In [1]:
PROTGPS_PARENT_DIR = "/home/shd-sun-lab/protgps" # point to the protgps local repo

In [2]:
import sys
import os
sys.path.append(PROTGPS_PARENT_DIR) # append the path of protgps
from argparse import Namespace
import pickle
from tqdm import tqdm
import pandas as pd
import torch 
from protgps.utils.loading import get_object
from tkinter import Tk, filedialog

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
COMPARTMENT_CLASSES = [
            "cytosol",
            "ER",
            "mitochondrion",
            "nucleus",
            "Excitatory Synapse",
            "Inhibitory Synapses"
]

def load_model(snargs):
    """
    Loads classifier model from args file
    """
    modelpath = snargs.model_path
    model = get_object(snargs.lightning_name, "lightning")(snargs)
    model = model.load_from_checkpoint(
        checkpoint_path = modelpath,
        strict=not snargs.relax_checkpoint_matching,
        **{"args": snargs},
    )
    return model

@torch.no_grad()
def predict_condensates(model, sequences, batch_size=1, round=True):
    scores = []
    for i in tqdm(range(0, len(sequences), batch_size), ncols=100):
        batch = sequences[ i : (i + batch_size)]
        out = model.model({"x": batch})    
        s = torch.sigmoid(out['logit']).to("cpu")
        scores.append(s)
    scores = torch.vstack(scores)
    if round:
        scores = torch.round(scores, decimals=3)
    return scores

In [8]:
# Use a file dialog to select the .args file
Tk().withdraw()  # Hide the main tkinter window
args_path = filedialog.askopenfilename(title="Select .args file", filetypes=[("Args files", "*.args")])

# Load args
args = Namespace(**pickle.load(open(args_path, 'rb')))

# Prompt to select the .ckpt file
ckpt_path = filedialog.askopenfilename(title="Select .ckpt file", filetypes=[("Checkpoint files", "*.ckpt")])
args.model_path = ckpt_path

# Set the pretrained hub directory manually (if static)
args.pretrained_hub_dir = "/home/shd-sun-lab/protgps/checkpoints/esm2"

# Load and prepare model
model = load_model(args)
model.eval()
model = model.to(device)

Using cache found in /home/shd-sun-lab/protgps/checkpoints/esm2/facebookresearch_esm_main


Using ESM hidden layers 6
Using ESM hidden layers 6


Using cache found in /home/shd-sun-lab/protgps/checkpoints/esm2/facebookresearch_esm_main


In [ ]:
#TITLE
sequences = [

]


# Add protein names corresponding to the sequences
protein_names = [


]

In [ ]:
#Calcium Homeostasis Genes
sequences = [
# CCG1_HUMAN
"MSQTKMLKVRVTLFCILAGIVLAMTAVVTDHWAVLSPHMEHHNTTCEAAHFGLWRICTKRIPMDDSKTCGPITLPGEKNCSYFRHFNPGESSEIFEFTTQKEYSISAAAIAIFSLGFIILGSLCVLLSLGKKRDYLLRPASMFYAFAGLCILVSVEVMRQSVKRMIDSEDTVWIEYYYSWSFACACAAFILLFLGGLALLLFSLPRMPRNPWESCMDAEPEH",
# CCG2_HUMAN
"MGLFDRGVQMLLTTVGAFAAFSLMTIAVGTDYWLYSRGVCKTKSVSENETSKKNEEVMTHSGLWRTCCLEGNFKGLCKQIDHFPEDADYEADTAEYFLRAVRASSIFPILSVILLFMGGLCIAASEFYKTRHNIILSAGIFFVSAGLSNIIGIIVYISANAGDPSKSDSKKNSYSYGWSFYFGALSFIIAEMVGVLAVHMFIDRHKQLRATARATDYLQASAITRIPSYRYRYQRRSRSSSRSTEPSHSRDASPVGIKGFNTLPSTEISMYTLSRDPLKAATTPTATYNSDRDNSFLQVHNCIQKENKDSLHSNTANRRTTPV",
# CCG3_HUMAN
"MRMCDRGIQMLITTVGAFAAFSLMTIAVGTDYWLYSRGVCRTKSTSDNETSRKNEEVMTHSGLWRTCCLEGAFRGVCKKIDHFPEDADYEQDTAEYLLRAVRASSVFPILSVTLLFFGGLCVAASEFHRSRHNVILSAGIFFVSAGLSNIIGIIVYISANAGDPGQRDSKKSYSYGWSFYFGAFSFIIAEIVGVVAVHIYIEKHQQLRAKSHSEFLKKSTFARLPPYRYRFRRRSSSRSTEPRSRDLSPISKGFHTIPSTDISMFTLSRDPSKITMGTLLNSDRDHAFLQFHNSTPKEFKESLHNNPANRRTTPV",
# CCG4_HUMAN
"MVRCDRGLQMLLTTAGAFAAFSLMAIAIGTDYWLYSSAHICNGTNLTMDDGPPPRRARGDLTHSGLWRVCCIEGIYKGHCFRINHFPEDNDYDHDSSEYLLRIVRASSVFPILSTILLLLGGLCIGAGRIYSRKNNIVLSAGILFVAAGLSNIIGIIVYISSNTGDPSDKRDEDKKNHYNYGWSFYFGALSFIVAETVGVLAVNIYIEKNKELRFKTKREFLKASSSSPYARMPSYRYRRRRSRSSSRSTEASPSRDVSPMGLKITGAIPMGELSMYTLSREPLKVTTAASYSPDQEASFLQVHDFFQQDLKEGFHVSMLNRRTTPV",
# CCG5_HUMAN
"MSACGRKALTLLSSVFAVCGLGLLGIAVSTDYWLYLEEGVIVPQNQSTEIKMSLHSGLWRVCFLAGEERGRCFTIEYVMPMNTQLTSESTVNVLKMIRSATPFPLVSLFFMFIGFILNNIGHIRPHRTILAFVSGIFFILSGLSLVVGLVLYISSINDEMLNRTKDAETYFNYKYGWSFAFAAISFLLTESAGVMSVYLFMKRYTAEDMYRPHPGFYRPRLSNCSDYSGQFLHPDAWVRGRSPSDISSEASLQMNSNYPALLKCPDYDQMSSSPC",
# CCG8_HUMAN
"MESLKRWNEERGLWCEKGVQVLLTTVGAFAAFGLMTIAISTDYWLYTRALICNTTNLTAGGDDGTPHRGGGGASEKKDPGGLTHSGLWRICCLEGLKRGVCVKINHFPEDTDYDHDSAEYLLRVVRASSIFPILSAILLLLGGVCVAASRVYKSKRNIILGAGILFVAAGLSNIIGVIVYISANAGEPGPKRDEEKKNHYSYGWSFYFGGLSFILAEVIGVLAVNIYIERSREAHCQSRSDLLKAGGGAGGSGGSGPSAILRLPSYRFRYRRRSRSSSRSSEPSPSRDASPGGPGGPGFASTDISMYTLSRDPSKGSVAAGLAGAGGGGGGAVGAFGGAAGGAGGGGGGGGGAGAERDRGGASGFLTLHNAFPKEAGGGVTVTVTGPPAPPAPAPPAPSAPAPGTLAKEAAASNTNTLNRKTTPV",
# CCG6_HUMAN
"MMWSNFFLQEENRRRGAAGRRRAHGQGRSGLTPEREGKVKLALLLAAVGATLAVLSVGTEFWVELNTYKANGSAVCEAAHLGLWKACTKRLWQADVPVDRDTCGPAELPGEANCTYFKFFTTGENARIFQRTTKKEVNLAAAVIAVLGLAVMALGCLCIIMVLSKGAEFLLRVGAVCFGLSGLLLLVSLEVFRHSVRALLQRVSPEPPPAPRLTYEYSWSLGCGVGAGLILLLGAGCFLLLTLPSWPWGSLCPKRGHRAT",
# CCG7_HUMAN
"MSHCSSRALTLLSSVFGACGLLLVGIAVSTDYWLYMEEGTVLPQNQTTEVKMALHAGLWRVCFFAGREKGRCVASEYFLEPEINLVTENTENILKTVRTATPFPMVSLFLVFTAFVISNIGHIRPQRTILAFVSGIFFILSGLSLVVGLVLYISSINDEVMNRPSSSEQYFHYRYGWSFAFAASSFLLKEGAGVMSVYLFTKRYAEEEMYRPHPAFYRPRLSDCSDYSGQFLQPEAWRRGRSPSDISSDVSIQMTQNYPPAIKYPDHLHISTSPC",
# CAC1A_HUMAN
"MARFGDEMPARYGGGGSGAAAGVVVGSGGGRGAGGSRQGGQPGAQRMYKQSMAQRARTMALYNPIPVRQNCLTVNRSLFLFSEDNVVRKYAKKITEWPPFEYMILATIIANCIVLALEQHLPDDDKTPMSERLDDTEPYFIGIFCFEAGIKIIALGFAFHKGSYLRNGWNVMDFVVVLTGILATVGTEFDLRTLRAVRVLRPLKLVSGIPSLQVVLKSIMKAMIPLLQIGLLLFFAILIFAIIGLEFYMGKFHTTCFEEGTDDIQGESPAPCGTEEPARTCPNGTKCQPYWEGPNNGITQFDNILFAVLTVFQCITMEGWTDLLYNSNDASGNTWNWLYFIPLIIIGSFFMLNLVLGVLSGEFAKERERVENRRAFLKLRRQQQIERELNGYMEWISKAEEVILAEDETDGEQRHPFDALRRTTIKKSKTDLLNPEEAEDQLADIASVGSPFARASIKSAKLENSTFFHKKERRMRFYIRRMVKTQAFYWTVLSLVALNTLCVAIVHYNQPEWLSDFLYYAEFIFLGLFMSEMFIKMYGLGTRPYFHSSFNCFDCGVIIGSIFEVIWAVIKPGTSFGISVLRALRLLRIFKVTKYWASLRNLVVSLLNSMKSIISLLFLLFLFIVVFALLGMQLFGGQFNFDEGTPPTNFDTFPAAIMTVFQILTGEDWNEVMYDGIKSQGGVQGGMVFSIYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEQEEEEAANQKLALQKAKEVAEVSPLSAANMSIAVKEQQKNQKPAKSVWEQRTSEMRKQNLLASREALYNEMDPDERWKAAYTRHLRPDMKTHLDRPLVVDPQENRNNNTNKSRAAEPTVDQRLGQQRAEDFLRKQARYHDRARDPSGSAGLDARRPWAGSQEAELSREGPYGRESDHHAREGSLEQPGFWEGEAERGKAGDPHRRHVHRQGGSRESRSGSPRTGADGEHRRHRAHRRPGEEGPEDKAERRARHREGSRPARGGEGEGEGPDGGERRRRHRHGAPATYEGDARREDKERRHRRRKENQGSGVPVSGPNLSTTRPIQQDLGRQDPPLAEDIDNMKNNKLATAESAAPHGSLGHAGLPQSPAKMGNSTDPGPMLAIPAMATNPQNAASRRTPNNPGNPSNPGPPKTPENSLIVTNPSGTQTNSAKTARKPDHTTVDIPPACPPPLNHTVVQVNKNANPDPLPKKEEEKKEEEEDDRGEDGPKPMPPYSSMFILSTTNPLRRLCHYILNLRYFEMCILMVIAMSSIALAAEDPVQPNAPRNNVLRYFDYVFTGVFTFEMVIKMIDLGLVLHQGAYFRDLWNILDFIVVSGALVAFAFTGNSKGKDINTIKSLRVLRVLRPLKTIKRLPKLKAVFDCVVNSLKNVFNILIVYMLFMFIFAVVAVQLFKGKFFHCTDESKEFEKDCRGKYLLYEKNEVKARDREWKKYEFHYDNVLWALLTLFTVSTGEGWPQVLKHSVDATFENQGPSPGYRMEMSIFYVVYFVVFPFFFVNIFVALIIITFQEQGDKMMEEYSLEKNERACIDFAISAKPLTRHMPQNKQSFQYRMWQFVVSPPFEYTIMAMIALNTIVLMMKFYGASVAYENALRVFNIVFTSLFSLECVLKVMAFGILNYFRDAWNIFDFVTVLGSITDILVTEFGNNFINLSFLRLFRAARLIKLLRQGYTIRILLWTFVQSFKALPYVCLLIAMLFFIYAIIGMQVFGNIGIDVEDEDSDEDEFQITEHNNFRTFFQALMLLFRSATGEAWHNIMLSCLSGKPCDKNSGILTRECGNEFAYFYFVSFIFLCSFLMLNLFVAVIMDNFEYLTRDSSILGPHHLDEYVRVWAEYDPAAWGRMPYLDMYQMLRHMSPPLGLGKKCPARVAYKRLLRMDLPVADDNTVHFNSTLMALIRTALDIKIAKGGADKQQMDAELRKEMMAIWPNLSQKTLDLLVTPHKSTDLTVGKIYAAMMIMEYYRQSKAKKLQAMREEQDRTPLMFQRMEPPSPTQEGGPGQNALPSTQLDPGGALMAHESGLKESPSWVTQRAQEMFQKTGTWSPEQGPPTDMPNSQPNSQSVEMREMGRDGYSDSEHYLPMEGQGRAASMPRLPAENQRRRGRPRGNNLSTISDTSPMKRSASVLGPKARRLDDYSLERVPPEENQRHHQRRRDRSHRASERSLGRYTDVDTGLGTDLSMTTQSGDLPSKERDQERGRPKDRKHRQHHHHHHHHHHPPPPDKDRYAQERPDHGRARARDQRWSRSPSEGREHMAHRQGSSSVSGSPAPSTSGTSTPRRGRRQLPQTPSTPRPHVSYSPVIRKAGGSGPPQQQQQQQQQQQQQAVARPGRAATSGPRRYPGPTAEPLAGDRPPTGGHSSGRSPRMERRVPGPARSESPRACRHGGARWPASGPHVSEGPPGPRHHGYYRGSDYDEADGPGSGGGEEAMAGAYDAPPPVRHASSGATGRSPRTPRASGPACASPSRHGRRLPNGYYPAHGLARPRGPGSRKGLHEPYSESDDDWC",
# CAC1B_HUMAN
"MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYKQSIAQRARTMALYNPIPVKQNCFTVNRSLFVFSEDNVVRKYAKRITEWPPFEYMILATIIANCIVLALEQHLPDGDKTPMSERLDDTEPYFIGIFCFEAGIKIIALGFVFHKGSYLRNGWNVMDFVVVLTGILATAGTDFDLRTLRAVRVLRPLKLVSGIPSLQVVLKSIMKAMVPLLQIGLLLFFAILMFAIIGLEFYMGKFHKACFPNSTDAEPVGDFPCGKEAPARLCEGDTECREYWPGPNFGITNFDNILFAILTVFQCITMEGWTDILYNTNDAAGNTWNWLYFIPLIIIGSFFMLNLVLGVLSGEFAKERERVENRRAFLKLRRQQQIERELNGYLEWIFKAEEVMLAEEDRNAEEKSPLDVLKRAATKKSRNDLIHAEEGEDRFADLCAVGSPFARASLKSGKTESSSYFRRKEKMFRFFIRRMVKAQSFYWVVLCVVALNTLCVAMVHYNQPRRLTTTLYFAEFVFLGLFLTEMSLKMYGLGPRSYFRSSFNCFDFGVIVGSVFEVVWAAIKPGSSFGISVLRALRLLRIFKVTKYWSSLRNLVVSLLNSMKSIISLLFLLFLFIVVFALLGMQLFGGQFNFQDETPTTNFDTFPAAILTVFQILTGEDWNAVMYHGIESQGGVSKGMFSSFYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEEEMEEAANQKLALQKAKEVAEVSPMSAANISIAARQQNSAKARSVWEQRASQLRLQNLRASCEALYSEMDPEERLRFATTRHLRPDMKTHLDRPLVVELGRDGARGPVGGKARPEAAEAPEGVDPPRRHHRHRDKDKTPAAGDQDRAEAPKAESGEPGAREERPRPHRSHSKEAAGPPEARSERGRGPGPEGGRRHHRRGSPEEAAEREPRRHRAHRHQDPSKECAGAKGERRARHRGGPRAGPREAESGEEPARRHRARHKAQPAHEAVEKETTEKEATEKEAEIVEADKEKELRNHQPREPHCDLETSGTVTVGPMHTLPSTCLQKVEEQPEDADNQRNVTRMGSQPPDPNTIVHIPVMLTGPLGEATVVPSGNVDLESQAEGKKEVEADDVMRSGPRPIVPYSSMFCLSPTNLLRRFCHYIVTMRYFEVVILVVIALSSIALAAEDPVRTDSPRNNALKYLDYIFTGVFTFEMVIKMIDLGLLLHPGAYFRDLWNILDFIVVSGALVAFAFSGSKGKDINTIKSLRVLRVLRPLKTIKRLPKLKAVFDCVVNSLKNVLNILIVYMLFMFIFAVIAVQLFKGKFFYCTDESKELERDCRGQYLDYEKEEVEAQPRQWKKYDFHYDNVLWALLTLFTVSTGEGWPMVLKHSVDATYEEQGPSPGYRMELSIFYVVYFVVFPFFFVNIFVALIIITFQEQGDKVMSECSLEKNERACIDFAISAKPLTRYMPQNRQSFQYKTWTFVVSPPFEYFIMAMIALNTVVLMMKFYDAPYEYELMLKCLNIVFTSMFSMECVLKIIAFGVLNYFRDAWNVFDFVTVLGSITDILVTEIAETNNFINLSFLRLFRAARLIKLLRQGYTIRILLWTFVQSFKALPYVCLLIAMLFFIYAIIGMQVFGNIALDDDTSINRHNNFRTFLQALMLLFRSATGEAWHEIMLSCLSNQACDEQANATECGSDFAYFYFVSFIFLCSFLMLNLFVAVIMDNFEYLTRDSSILGPHHLDEFIRVWAEYDPAACGRISYNDMFEMLKHMSPPLGLGKKCPARVAYKRLVRMNMPISNEDMTVHFTSTLMALIRTALEIKLAPAGTKQHQCDAELRKEISVVWANLPQKTLDLLVPPHKPDEMTVGKVYAALMIFDFYKQNKTTRDQMQQAPGGLSQMGPVSLFHPLKATLEQTQPAVLRGARVFLRQKSSTSLSNGGAIQNQESGIKESVSWGTQRTQDAPHEARPPLERGHSTEIPVGRSGALAVDVQMQSITRRGPDGEPQPGLESQGRAASMPRLAAETQPVTDASPMKRSISTLAQRPRGTHLCSTTPDRPPPSQASSHHHHHRCHRRRDRKQRSLEKGPSLSADMDGAPSSAVGPGLPPGEGPTGCRRERERRQERGRSQERRQPSSSSSEKQRFYSCDRFGGREPPKPKPSLSSHPTSPTAGQEPGPHPQGSGSVNGSPLLSTSGASTPGRGGRRQLPQTPLTPRPSITYKTANSSPIHFAGAQTSLPAFSPGRLSRGLSEHNALLQRDPLSQPLAPGSRIGSDPYLGQRLDSEASVHALPEDTLTFEEAVATNSGRSSRTSYVSSLTSQSHPLRRVPNGYHCTLGLSSGGRARHSYHHPDQDHWC",
# CAC1C_HUMAN
"MVNENTRMYIPEENHQGSNYGSPRPAHANMNANAAAGLAPEHIPTPGAALSWQAAIDAARQAKLMGSAGNATISTVSSTQRKRQQYGKPKKQGSTTATRPPRALLCLTLKNPIRRACISIVEWKPFEIIILLTIFANCVALAIYIPFPEDDSNATNSNLERVEYLFLIIFTVEAFLKVIAYGLLFHPNAYLRNGWNLLDFIIVVVGLFSAILEQATKADGANALGGKGAGFDVKALRAFRVLRPLRLVSGVPSLQVVLNSIIKAMVPLLHIALLVLFVIIIYAIIGLELFMGKMHKTCYNQEGIADVPAEDDPSPCALETGHGRQCQNGTVCKPGWDGPKHGITNFDNFAFAMLTVFQCITMEGWTDVLYWVNDAVGRDWPWIYFVTLIIIGSFFVLNLVLGVLSGEFSKEREKAKARGDFQKLREKQQLEEDLKGYLDWITQAEDIDPENEDEGMDEEKPRNMSMPTSETESVNTENVAGGDIEGENCGARLAHRISKSKFSRYWRRWNRFCRRKCRAAVKSNVFYWLVIFLVFLNTLTIASEHYNQPNWLTEVQDTANKALLALFTAEMLLKMYSLGLQAYFVSLFNRFDCFVVCGGILETILVETKIMSPLGISVLRCVRLLRIFKITRYWNSLSNLVASLLNSVRSIASLLLLLFLFIIIFSLLGMQLFGGKFNFDEMQTRRSTFDNFPQSLLTVFQILTGEDWNSVMYDGIMAYGGPSFPGMLVCIYFIILFICGNYILLNVFLAIAVDNLADAESLTSAQKEEEEEKERKKLARTASPEKKQELVEKPAVGESKEEKIELKSITADGESPPATKINMDDLQPNENEDKSPYPNPETTGEEDEEEPEMPVGPRPRPLSELHLKEKAVPMPEASAFFIFSSNNRFRLQCHRIVNDTIFTNLILFFILLSSISLAAEDPVQHTSFRNHILFYFDIVFTTIFTIEIALKILGNADYVFTSIFTLEIILKMTAYGAFLHKGSFCRNYFNILDLLVVSVSLISFGIQSSAINVVKILRVLRVLRPLRAINRAKGLKHVVQCVFVAIRTIGNIVIVTTLLQFMFACIGVQLFKGKLYTCSDSSKQTEAECKGNYITYKDGEVDHPIIQPRSWENSKFDFDNVLAAMMALFTVSTFEGWPELLYRSIDSHTEDKGPIYNYRVEISIFFIIYIIIIAFFMMNIFVGFVIVTFQEQGEQEYKNCELDKNQRQCVEYALKARPLRRYIPKNQHQYKVWYVVNSTYFEYLMFVLILLNTICLAMQHYGQSCLFKIAMNILNMLFTGLFTVEMILKLIAFKPKGYFSDPWNVFDFLIVIGSIIDVILSETNHYFCDAWNTFDALIVVGSIVDIAITEVNPAEHTQCSPSMNAEENSRISITFFRLFRVMRLVKLLSRGEGIRTLLWTFIKSFQALPYVALLIVMLFFIYAVIGMQVFGKIALNDTTEINRNNNFQTFPQAVLLLFRCATGEAWQDIMLACMPGKKCAPESEPSNSTEGETPCGSSFAVFYFISFYMLCAFLIINLFVAVIMDNFDYLTRDWSILGPHHLDEFKRIWAEYDPEAKGRIKHLDVVTLLRRIQPPLGFGKLCPHRVACKRLVSMNMPLNSDGTVMFNATLFALVRTALRIKTEGNLEQANEELRAIIKKIWKRTSMKLLDQVVPPAGDDEVTVGKFYATFLIQEYFRKFKKRKEQGLVGKPSQRNALSLQAGLRTLHDIGPEIRRAISGDLTAEEELDKAMKEAVSAASEDDIFRRAGGLFGNHVSYYQSDGRSAFPQTFTTQRPLHINKAGSSQGDTESPSHEKLVDSTFTPSSYSSTGSNANINNANNTALGRLPRPAGYPSTVSTVEGHGPPLSPAIRVQEVAWKLSSNRERHVPMCEDLELRRDSGSAGTQAHCLLLRKANPSRCHSRESQAAMAGQEETSQDETYEVKMNHDTEACSEPSLLSTEMLSYQDDENRQLTLPEEDKRDIRQSPKRGFLRSASLGRRASFHLECLKRQKDRGGDISQKTVLPLHLVHHQALAVAGLSPLLQRSHSPASFPRPFATPPATPGSRGWPPQPVPTLRLEGVESSEKLNSSFPSIHCGSWAETTPGGGGSSAARRVRPVSLMVPSQAGAPGRQFHGSASSLVEAVLISEGLGQFAQDPKFIEVTTQELADACDMTIEEMESAADNILSGGAPQSPNGALLPFVNCRDAGQDRAGGEEDAGCVRARGRPSEEELQDSRVYVSSL",
# CAC1D_HUMAN
"MMMMMMMKKMQHQRQQQADHANEANYARGTRLPLSGEGPTSQPNSSKQTVLSWQAAIDAARQAKAAQTMSTSAPPPVGSLSQRKRQQYAKSKKQGNSSNSRPARALFCLSLNNPIRRACISIVEWKPFDIFILLAIFANCVALAIYIPFPEDDSNSTNHNLEKVEYAFLIIFTVETFLKIIAYGLLLHPNAYVRNGWNLLDFVIVIVGLFSVILEQLTKETEGGNHSSGKSGGFDVKALRAFRVLRPLRLVSGVPSLQVVLNSIIKAMVPLLHIALLVLFVIIIYAIIGLELFIGKMHKTCFFADSDIVAEEDPAPCAFSGNGRQCTANGTECRSGWVGPNGGITNFDNFAFAMLTVFQCITMEGWTDVLYWMNDAMGFELPWVYFVSLVIFGSFFVLNLVLGVLSGEFSKEREKAKARGDFQKLREKQQLEEDLKGYLDWITQAEDIDPENEEEGGEEGKRNTSMPTSETESVNTENVSGEGENRGCCGSLCQAISKSKLSRRWRRWNRFNRRRCRAAVKSVTFYWLVIVLVFLNTLTISSEHYNQPDWLTQIQDIANKVLLALFTCEMLVKMYSLGLQAYFVSLFNRFDCFVVCGGITETILVELEIMSPLGISVFRCVRLLRIFKVTRHWTSLSNLVASLLNSMKSIASLLLLLFLFIIIFSLLGMQLFGGKFNFDETQTKRSTFDNFPQALLTVFQILTGEDWNAVMYDGIMAYGGPSSSGMIVCIYFIILFICGNYILLNVFLAIAVDNLADAESLNTAQKEEAEEKERKKIARKESLENKKNNKPEVNQIANSDNKVTIDDYREEDEDKDPYPPCDVPVGEEEEEEEEDEPEVPAGPRPRRISELNMKEKIAPIPEGSAFFILSKTNPIRVGCHKLINHHIFTNLILVFIMLSSAALAAEDPIRSHSFRNTILGYFDYAFTAIFTVEILLKMTTFGAFLHKGAFCRNYFNLLDMLVVGVSLVSFGIQSSAISVVKILRVLRVLRPLRAINRAKGLKHVVQCVFVAIRTIGNIMIVTTLLQFMFACIGVQLFKGKFYRCTDEAKSNPEECRGLFILYKDGDVDSPVVRERIWQNSDFNFDNVLSAMMALFTVSTFEGWPALLYKAIDSNGENIGPIYNHRVEISIFFIIYIIIVAFFMMNIFVGFVIVTFQEQGEKEYKNCELDKNQRQCVEYALKARPLRRYIPKNPYQYKFWYVVNSSPFEYMMFVLIMLNTLCLAMQHYEQSKMFNDAMDILNMVFTGVFTVEMVLKVIAFKPKGYFSDAWNTFDSLIVIGSIIDVALSEADPTESENVPVPTATPGNSEESNRISITFFRLFRVMRLVKLLSRGEGIRTLLWTFIKSFQALPYVALLIAMLFFIYAVIGMQMFGKVAMRDNNQINRNNNFQTFPQAVLLLFRCATGEAWQEIMLACLPGKLCDPESDYNPGEEYTCGSNFAIVYFISFYMLCAFLIINLFVAVIMDNFDYLTRDWSILGPHHLDEFKRIWSEYDPEAKGRIKHLDVVTLLRRIQPPLGFGKLCPHRVACKRLVAMNMPLNSDGTVMFNATLFALVRTALKIKTEGNLEQANEELRAVIKKIWKKTSMKLLDQVVPPAGDDEVTVGKFYATFLIQDYFRKFKKRKEQGLVGKYPAKNTTIALQAGLRTLHDIGPEIRRAISCDLQDDEPEETKREEEDDVFKRNGALLGNHVNHVNSDRRDSLQQTNTTHRPLHVQRPSIPPASDTEKPLFPPAGNSVCHNHHNHNSIGKQVPTSTNANLNNANMSKAAHGKRPSIGNLEHVSENGHHSSHKHDREPQRRSSVKRTRYYETYIRSDSGDEQLPTICREDPEIHGYFRDPHCLGEQEYFSSEECYEDDSSPTWSRQNYGYYSRYPGRNIDSERPRGYHHPQGFLEDDDSPVCYDSRRSPRRRLLPPTPASHRRSSFNFECLRRQSSQEEVPSSPIFPHRTALPLHLMQQQIMAVAGLDSSKAQKYSPSHSTRSWATPPATPPYRDWTPCYTPLIQVEQSEALDQVNGSLPSLHRSSWYTDEPDISYRTFTPASLTVPSSFRNKNSDKQRSADSLVEAVLISEGLGRYARDPKFVSATKHEIADACDLTIDEMESAASTLLNGNVRPRANGDVGPLSHRQDYELQDFGPGYSDEEPDPGRDEEDLADEMICITTL",
# CAC1E_HUMAN
"MARFGEAVVARPGSGDGDSDQSRNRQGTPVPASGQAAAYKQTKAQRARTMALYNPIPVRQNCFTVNRSLFIFGEDNIVRKYAKKLIDWPPFEYMILATIIANCIVLALEQHLPEDDKTPMSRRLEKTEPYFIGIFCFEAGIKIVALGFIFHKGSYLRNGWNVMDFIVVLSGILATAGTHFNTHVDLRTLRAVRVLRPLKLVSGIPSLQIVLKSIMKAMVPLLQIGLLLFFAILMFAIIGLEFYSGKLHRACFMNNSGILEGFDPPHPCGVQGCPAGYECKDWIGPNDGITQFDNILFAVLTVFQCITMEGWTTVLYNTNDALGATWNWLYFIPLIIIGSFFVLNLVLGVLSGEFAKERERVENRRAFMKLRRQQQIERELNGYRAWIDKAEEVMLAEENKNAGTSALEVLRRATIKRSRTEAMTRDSSDEHCVDISSVGTPLARASIKSAKVDGVSYFRHKERLLRISIRHMVKSQVFYWIVLSLVALNTACVAIVHHNQPQWLTHLLYYAEFLFLGLFLLEMSLKMYGMGPRLYFHSSFNCFDFGVTVGSIFEVVWAIFRPGTSFGISVLRALRLLRIFKITKYWASLRNLVVSLMSSMKSIISLLFLLFLFIVVFALLGMQLFGGRFNFNDGTPSANFDTFPAAIMTVFQILTGEDWNEVMYNGIRSQGGVSSGMWSAIYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEQEEEEAFNQKHALQKAKEVSPMSAPNMPSIERDRRRRHHMSMWEPRSSHLRERRRRHHMSVWEQRTSQLRKHMQMSSQEALNREEAPTMNPLNPLNPLSSLNPLNAHPSLYRRPRAIEGLALGLALEKFEEERISRGGSLKGDGGDRSSALDNQRTPLSLGQREPPWLARPCHGNCDPTQQEAGGGEAVVTFEDRARHRQSQRRSRHRRVRTEGKESSSASRSRSASQERSLDEAMPTEGEKDHELRGNHGAKEPTIQEERAQDLRRTNSLMVSRGSGLAGGLDEADTPLVLPHPELEVGKHVVLTEQEPEGSSEQALLGNVQLDMGRVISQSEPDLSCITANTDKATTESTSVTVAIPDVDPLVDSTVVHISNKTDGEASPLKEAEIREDEEEVEKKKQKKEKRETGKAMVPHSSMFIFSTTNPIRRACHYIVNLRYFEMCILLVIAASSIALAAEDPVLTNSERNKVLRYFDYVFTGVFTFEMVIKMIDQGLILQDGSYFRDLWNILDFVVVVGALVAFALANALGTNKGRDIKTIKSLRVLRVLRPLKTIKRLPKLKAVFDCVVTSLKNVFNILIVYKLFMFIFAVIAVQLFKGKFFYCTDSSKDTEKECIGNYVDHEKNKMEVKGREWKRHEFHYDNIIWALLTLFTVSTGEGWPQVLQHSVDVTEEDRGPSRSNRMEMSIFYVVYFVVFPFFFVNIFVALIIITFQEQGDKMMEECSLEKNERACIDFAISAKPLTRYMPQNRHTFQYRVWHFVVSPSFEYTIMAMIALNTVVLMMKYYSAPCTYELALKYLNIAFTMVFSLECVLKVIAFGFLNYFRDTWNIFDFITVIGSITEIILTDSKLVNTSGFNMSFLKLFRAARLIKLLRQGYTIRILLWTFVQSFKALPYVCLLIAMLFFIYAIIGMQVFGNIKLDEESHINRHNNFRSFFGSLMLLFRSATGEAWQEIMLSCLGEKGCEPDTTAPSGQNENERCGTDLAYVYFVSFIFFCSFLMLNLFVAVIMDNFEYLTRDSSILGPHHLDEFVRVWAEYDRAACGRIHYTEMYEMLTLMSPPLGLGKRCPSKVAYKRLVLMNMPVAEDMTVHFTSTLMALIRTALDIKIAKGGADRQQLDSELQKETLAIWPHLSQKMLDLLVPMPKASDLTVGKIYAAMMIMDYYKQSKVKKQRQQLEEQKNAPMFQRMEPSSLPQEIIANAKALPYLQQDPVSGLSGRSGYPSMSPLSPQDIFQLACMDPADDGQFQERQSLEPEVSELKSVQPSNHGIYLPSDTQEHAGSGRASSMPRLTVDPQVVTDPSSMRRSFSTIRDKRSNSSWLEEFSMERSSENTYKSRRRSYHSSLRLSAHRLNSDSGHKSDTHRSGGRERGRSKERKHLLSPDVSRCNSEERGTQADWESPERRQSRSPSEGRSQTPNRQGTGSLSESSIPSVSDTSTPRRSRRQLPPVPPKPRPLLSYSSLIRHAGSISPPADGSEEGSPLTSQALESNNACLTESSNSPHPQQSQHASPQRYISEPYLALHEDSHASDCGEEETLTFEAAVATSLGRSNTIGSAPPLRHSWQMPNGHYRRRRRGGPGPGMMCGAVNNLLSDTEEDDKC",
# CAC1F_HUMAN
"MSESEGGKDTTPEPSPANGAGPGPEWGLCPGPPAVEGESSGASGLGTPKRRNQHSKHKTVAVASAQRSPRALFCLTLANPLRRSCISIVEWKPFDILILLTIFANCVALGVYIPFPEDDSNTANHNLEQVEYVFLVIFTVETVLKIVAYGLVLHPSAYIRNGWNLLDFIIVVVGLFSVLLEQGPGRPGDAPHTGGKPGGFDVKALRAFRVLRPLRLVSGVPSLHIVLNSIMKALVPLLHIALLVLFVIIIYAIIGLELFLGRMHKTCYFLGSDMEAEEDPSPCASSGSGRACTLNQTECRGRWPGPNGGITNFDNFFFAMLTVFQCVTMEGWTDVLYWMQDAMGYELPWVYFVSLVIFGSFFVLNLVLGVLSGEFSKEREKAKARGDFQKQREKQQMEEDLRGYLDWITQAEELDMEDPSADDNLGSMAEEGRAGHRPQLAELTNRRRGRLRWFSHSTRSTHSTSSHASLPASDTGSMTETQGDEDEEEGALASCTRCLNKIMKTRVCRRLRRANRVLRARCRRAVKSNACYWAVLLLVFLNTLTIASEHHGQPVWLTQIQEYANKVLLCLFTVEMLLKLYGLGPSAYVSSFFNRFDCFVVCGGILETTLVEVGAMQPLGISVLRCVRLLRIFKVTRHWASLSNLVASLLNSMKSIASLLLLLFLFIIIFSLLGMQLFGGKFNFDQTHTKRSTFDTFPQALLTVFQILTGEDWNVVMYDGIMAYGGPFFPGMLVCIYFIILFICGNYILLNVFLAIAVDNLASGDAGTAKDKGGEKSNEKDLPQENEGLVPGVEKEEEEGARREGADMEEEEEEEEEEEEEEEEEGAGGVELLQEVVPKEKVVPIPEGSAFFCLSQTNPLRKGCHTLIHHHVFTNLILVFIILSSVSLAAEDPIRAHSFRNHILGYFDYAFTSIFTVEILLKMTVFGAFLHRGSFCRSWFNMLDLLVVSVSLISFGIHSSAISVVKILRVLRVLRPLRAINRAKGLKHVVQCVFVAIRTIGNIMIVTTLLQFMFACIGVQLFKGKFYTCTDEAKHTPQECKGSFLVYPDGDVSRPLVRERLWVNSDFNFDNVLSAMMALFTVSTFEGWPALLYKAIDAYAEDHGPIYNYRVEISVFFIVYIIIIAFFMMNIFVGFVIITFRAQGEQEYQNCELDKNQRQCVEYALKAQPLRRYIPKNPHQYRVWATVNSAAFEYLMFLLILLNTVALAMQHYEQTAPFNYAMDILNMVFTGLFTIEMVLKIIAFKPKHYFTDAWNTFDALIVVGSIVDIAVTEVNNGGHLGESSEDSSRISITFFRLFRVMRLVKLLSKGEGIRTLLWTFIKSFQALPYVALLIAMIFFIYAVIGMQMFGKVALQDGTQINRNNNFQTFPQAVLLLFRCATGEAWQEIMLASLPGNRCDPESDFGPGEEFTCGSNFAIAYFISFFMLCAFLIINLFVAVIMDNFDYLTRDWSILGPHHLDEFKRIWSEYDPGAKGRIKHLDVVALLRRIQPPLGFGKLCPHRVACKRLVAMNMPLNSDGTVTFNATLFALVRTSLKIKTEGNLEQANQELRIVIKKIWKRMKQKLLDEVIPPPDEEEVTVGKFYATFLIQDYFRKFRRRKEKGLLGNDAAPSTSSALQAGLRSLQDLGPEMRQALTCDTEEEEEEGQEGVEEEDEKDLETNKATMVSQPSARRGSGISVSLPVGDRLPDSLSFGPSDDDRGTPTSSQPSVPQAGSNTHRRGSGALIFTIPEEGNSQPKGTKGQNKQDEDEEVPDRLSYLDEQAGTPPCSVLLPPHRAQRYMDGHLVPRRRLLPPTPAGRKPSFTIQCLQRQGSCEDLPIPGTYHRGRNSGPNRAQGSWATPPQRGRLLYAPLLLVEEGAAGEGYLGRSSGPLRTFTCLHVPGTHSDPSHGKRGSADSLVEAVLISEGLGLFARDPRFVALAKQEIADACRLTLDEMDNAASDLLAQGTSSLYSDEESILSRFDEEDLGDEMACVHAL",
# CAC1G_HUMAN
"MDEEEDGAGAEESGQPRSFMRLNDLSGAGGRPGPGSAEKDPGSADSEAEGLPYPALAPVVFFYLSQDSRPRSWCLRTVCNPWFERISMLVILLNCVTLGMFRPCEDIACDSQRCRILQAFDDFIFAFFAVEMVVKMVALGIFGKKCYLGDTWNRLDFFIVIAGMLEYSLDLQNVSFSAVRTVRVLRPLRAINRVPSMRILVTLLLDTLPMLGNVLLLCFFVFFIFGIVGVQLWAGLLRNRCFLPENFSLPLSVDLERYYQTENEDESPFICSQPRENGMRSCRSVPTLRGDGGGGPPCGLDYEAYNSSSNTTCVNWNQYYTNCSAGEHNPFKGAINFDNIGYAWIAIFQVITLEGWVDIMYFVMDAHSFYNFIYFILLIIVGSFFMINLCLVVIATQFSETKQRESQLMREQRVRFLSNASTLASFSEPGSCYEELLKYLVYILRKAARRLAQVSRAAGVRVGLLSSPAPLGGQETQPSSSCSRSHRRLSVHHLVHHHHHHHHHYHLGNGTLRAPRASPEIQDRDANGSRRLMLPPPSTPALSGAPPGGAESVHSFYHADCHLEPVRCQAPPPRSPSEASGRTVGSGKVYPTVHTSPPPETLKEKALVEVAASSGPPTLTSLNIPPGPYSSMHKLLETQSTGACQSSCKISSPCLKADSGACGPDSCPYCARAGAGEVELADREMPDSDSEAVYEFTQDAQHSDLRDPHSRRQRSLGPDAEPSSVLAFWRLICDTFRKIVDSKYFGRGIMIAILVNTLSMGIEYHEQPEELTNALEISNIVFTSLFALEMLLKLLVYGPFGYIKNPYNIFDGVIVVISVWEIVGQQGGGLSVLRTFRLMRVLKLVRFLPALQRQLVVLMKTMDNVATFCMLLMLFIFIFSILGMHLFGCKFASERDGDTLPDRKNFDSLLWAIVTVFQILTQEDWNKVLYNGMASTSSWAALYFIALMTFGNYVLFNLLVAILVEGFQAEEISKREDASGQLSCIQLPVDSQGGDANKSESEPDFFSPSLDGDGDRKKCLALVSLGEHPELRKSLLPPLIIHTAATPMSLPKSTSTGLGEALGPASRRTSSSGSAEPGAAHEMKSPPSARSSPHSPWSAASSWTSRRSSRNSLGRAPSLKRRSPSGERRSLLSGEGQESQDEEESSEEERASPAGSDHRHRGSLEREAKSSFDLPDTLQVPGLHRTASGRGSASEHQDCNGKSASGRLARALRPDDPPLDGDDADDEGNLSKGERVRAWIRARLPACCLERDSWSAYIFPPQSRFRLLCHRIITHKMFDHVVLVIIFLNCITIAMERPKIDPHSAERIFLTLSNYIFTAVFLAEMTVKVVALGWCFGEQAYLRSSWNVLDGLLVLISVIDILVSMVSDSGTKILGMLRVLRLLRTLRPLRVISRAQGLKLVVETLMSSLKPIGNIVVICCAFFIIFGILGVQLFKGKFFVCQGEDTRNITNKSDCAEASYRWVRHKYNFDNLGQALMSLFVLASKDGWVDIMYDGLDAVGVDQQPIMNHNPWMLLYFISFLLIVAFFVLNMFVGVVVENFHKCRQHQEEEEARRREEKRLRRLEKKRRNLMLDDVIASGSSASAASEAQCKPYYSDYSRFRLLVHHLCTSHYLDLFITGVIGLNVVTMAMEHYQQPQILDEALKICNYIFTVIFVLESVFKLVAFGFRRFFQDRWNQLDLAIVLLSIMGITLEEIEVNASLPINPTIIRIMRVLRIARVLKLLKMAVGMRALLDTVMQALPQVGNLGLLFMLLFFIFAALGVELFGDLECDETHPCEGLGRHATFRNFGMAFLTLFRVSTGDNWNGIMKDTLRDCDQESTCYNTVISPIYFVSFVLTAQFVLVNVVIAVLMKHLEESNKEAKEEAELEAELELEMKTLSPQPHSPLGSPFLWPGVEGPDSPDSPKPGALHPAAHARSASHFSLEHPTDRQLFDTISLLIQGSLEWELKLMDELAGPGGQPSAFPSAPSLGGSDPQIPLAEMEALSLTSEIVSEPSCSLALTDDSLPDDMHTLLLSALESNMQPHPTELPGPDLLTVRKSGVSRTHSLPNDSYMCRHGSTAEGPLGHRGWGLPKAQSGSVLSVHSQPADTSYILQLPKDAPHLLQPHSAPTWGTIPKLPPPGRSPLAQRPLRRQAAIRTDSLDVQGLGSREDLLAEVSGPSPPLARAYSFWGQSSTQAQQHSRSHSKISKHMTPPAPCPGPEPNWGKGPPETRSSLELDTELSWISGDLLPPGGQEEPPSPRDLKKCYSVEAQSCQRRPTSWLDEQRRHSIAVSCLDSGSQPHLGTDPSNLGGQPLGGPGSRPKKKLSPPSITIDPPESQGPRTPPSPGICLRRRAPSSDSKDPLASGPPDSMAASPSPKKDVLSLSGLSSDPADLDP",
# CAC1H_HUMAN
"MTEGARAADEVRVPLGAPPPGPAALVGASPESPGAPGREAERGSELGVSPSESPAAERGAELGADEEQRVPYPALAATVFFCLGQTTRPRSWCLRLVCNPWFEHVSMLVIMLNCVTLGMFRPCEDVECGSERCNILEAFDAFIFAFFAVEMVIKMVALGLFGQKCYLGDTWNRLDFFIVVAGMMEYSLDGHNVSLSAIRTVRVLRPLRAINRVPSMRILVTLLLDTLPMLGNVLLLCFFVFFIFGIVGVQLWAGLLRNRCFLDSAFVRNNNLTFLRPYYQTEEGEENPFICSSRRDNGMQKCSHIPGRRELRMPCTLGWEAYTQPQAEGVGAARNACINWNQYYNVCRSGDSNPHNGAINFDNIGYAWIAIFQVITLEGWVDIMYYVMDAHSFYNFIYFILLIIVGSFFMINLCLVVIATQFSETKQRESQLMREQRARHLSNDSTLASFSEPGSCYEELLKYVGHIFRKVKRRSLRLYARWQSRWRKKVDPSAVQGQGPGHRQRRAGRHTASVHHLVYHHHHHHHHHYHFSHGSPRRPGPEPGACDTRLVRAGAPPSPPSPGRGPPDAESVHSIYHADCHIEGPQERARVAHAAATAAASLRLATGLGTMNYPTILPSGVGSGKGSTSPGPKGKWAGGPPGTGGHGPLSLNSPDPYEKIPHVVGEHGLGQAPGHLSGLSVPCPLPSPPAGTLTCELKSCPYCTRALEDPEGELSGSESGDSDGRGVYEFTQDVRHGDRWDPTRPPRATDTPGPGPGSPQRRAQQRAAPGEPGWMGRLWVTFSGKLRRIVDSKYFSRGIMMAILVNTLSMGVEYHEQPEELTNALEISNIVFTSMFALEMLLKLLACGPLGYIRNPYNIFDGIIVVISVWEIVGQADGGLSVLRTFRLLRVLKLVRFLPALRRQLVVLVKTMDNVATFCTLLMLFIFIFSILGMHLFGCKFSLKTDTGDTVPDRKNFDSLLWAIVTVFQILTQEDWNVVLYNGMASTSSWAALYFVALMTFGNYVLFNLLVAILVEGFQAEGDANRSDTDEDKTSVHFEEDFHKLRELQTTELKMCSLAVTPNGHLEGRGSLSPPLIMCTAATPMPTPKSSPFLDAAPSLPDSRRGSSSSGDPPLGDQKPPASLRSSPCAPWGPSGAWSSRRSSWSSLGRAPSLKRRGQCGERESLLSGEGKGSTDDEAEDGRAAPGPRATPLRRAESLDPRPLRPAALPPTKCRDRDGQVVALPSDFFLRIDSHREDAAELDDDSEDSCCLRLHKVLEPYKPQWCRSREAWALYLFSPQNRFRVSCQKVITHKMFDHVVLVFIFLNCVTIALERPDIDPGSTERVFLSVSNYIFTAIFVAEMMVKVVALGLLSGEHAYLQSSWNLLDGLLVLVSLVDIVVAMASAGGAKILGVLRVLRLLRTLRPLRVISRAPGLKLVVETLISSLRPIGNIVLICCAFFIIFGILGVQLFKGKFYYCEGPDTRNISTKAQCRAAHYRWVRRKYNFDNLGQALMSLFVLSSKDGWVNIMYDGLDAVGVDQQPVQNHNPWMLLYFISFLLIVSFFVLNMFVGVVVENFHKCRQHQEAEEARRREEKRLRRLERRRRSTFPSPEAQRRPYYADYSPTRRSIHSLCTSHYLDLFITFIICVNVITMSMEHYNQPKSLDEALKYCNYVFTIVFVFEAALKLVAFGFRRFFKDRWNQLDLAIVLLSLMGITLEEIEMSAALPINPTIIRIMRVLRIARVLKLLKMATGMRALLDTVVQALPQVGNLGLLFMLLFFIYAALGVELFGRLECSEDNPCEGLSRHATFSNFGMAFLTLFRVSTGDNWNGIMKDTLRECSREDKHCLSYLPALSPVYFVTFVLVAQFVLVNVVVAVLMKHLEESNKEAREDAELDAEIELEMAQGPGSARRVDADRPPLPQESPGARDAPNLVARKVSVSRMLSLPNDSYMFRPVVPASAPHPRPLQEVEMETYGAGTPLGSVASVHSPPAESCASLQIPLAVSSPARSGEPLHALSPRGTARSPSLSRLLCRQEAVHTDSLEGKIDSPRDTLDPAEPGEKTPVRPVTQGGSLQSPPRSPRPASVRTRKHTFGQRCVSSRPAAPGGEEAEASDPADEEVSHITSSACPWQPTAEPHGPEASPVAGGERDLRRLYSVDAQGFLDKPGRADEQWRPSAELGSGEPGEAKAWGPEAEPALGARRKKKMSPPCISVEPPAEDEGSARPSAAEGGSTTLRRRTPSCEATPHRDSLEPTEGSGAGGDPAAKGERWGQASCRAEHLTVPSFAFEPLDLGVPSGDPFLDGSHSVTPESRASSSGAIVPLEPPESEPPMPVGDPPEKRRGLYLTVPQCPLEKPGSPSATPAPGGGADDPV",
# CAC1I_HUMAN
"MAESASPPSSSAAAPAAEPGVTTEQPGPRSPPSSPPGLEEPLDGADPHVPHPDLAPIAFFCLRQTTSPRNWCIKMVCNPWFECVSMLVILLNCVTLGMYQPCDDMDCLSDRCKILQVFDDFIFIFFAMEMVLKMVALGIFGKKCYLGDTWNRLDFFIVMAGMVEYSLDLQNINLSAIRTVRVLRPLKAINRVPSMRILVNLLLDTLPMLGNVLLLCFFVFFIFGIIGVQLWAGLLRNRCFLEENFTIQGDVALPPYYQPEEDDEMPFICSLSGDNGIMGCHEIPPLKEQGRECCLSKDDVYDFGAGRQDLNASGLCVNWNRYYNVCRTGSANPHKGAINFDNIGYAWIVIFQVITLEGWVEIMYYVMDAHSFYNFIYFILLIIVGSFFMINLCLVVIATQFSETKQREHRLMLEQRQRYLSSSTVASYAEPGDCYEEIFQYVCHILRKAKRRALGLYQALQSRRQALGPEAPAPAKPGPHAKEPRHYHGKTKGQGDEGRHLGSRHCQTLHGPASPGNDHSGRELCPQHSPLDATPHTLVQPIPATLASDPASCPCCQHEDGRRPSGLGSTDSGQEGSGSGSSAGGEDEADGDGARSSEDGASSELGKEEEEEEQADGAVWLCGDVWRETRAKLRGIVDSKYFNRGIMMAILVNTVSMGIEHHEQPEELTNILEICNVVFTSMFALEMILKLAAFGLFDYLRNPYNIFDSIIVIISIWEIVGQADGGLSVLRTFRLLRVLKLVRFMPALRRQLVVLMKTMDNVATFCMLLMLFIFIFSILGMHIFGCKFSLRTDTGDTVPDRKNFDSLLWAIVTVFQILTQEDWNVVLYNGMASTSPWASLYFVALMTFGNYVLFNLLVAILVEGFQAEGDANRSYSDEDQSSSNIEEFDKLQEGLDSSGDPKLCPIPMTPNGHLDPSLPLGGHLGPAGAAGPAPRLSLQPDPMLVALGSRKSSVMSLGRMSYDQRSLSSSRSSYYGPWGRSAAWASRRSSWNSLKHKPPSAEHESLLSAERGGGARVCEVAADEGPPRAAPLHTPHAHHIHHGPHLAHRHRHHRRTLSLDNRDSVDLAELVPAVGAHPRAAWRAAGPAPGHEDCNGRMPSIAKDVFTKMGDRGDRGEDEEEIDYTLCFRVRKMIDVYKPDWCEVREDWSVYLFSPENRFRVLCQTIIAHKLFDYVVLAFIFLNCITIALERPQIEAGSTERIFLTVSNYIFTAIFVGEMTLKVVSLGLYFGEQAYLRSSWNVLDGFLVFVSIIDIVVSLASAGGAKILGVLRVLRLLRTLRPLRVISRAPGLKLVVETLISSLKPIGNIVLICCAFFIIFGILGVQLFKGKFYHCLGVDTRNITNRSDCMAANYRWVHHKYNFDNLGQALMSLFVLASKDGWVNIMYNGLDAVAVDQQPVTNHNPWMLLYFISFLLIVSFFVLNMFVGVVVENFHKCRQHQEAEEARRREEKRLRRLEKKRRKAQRLPYYATYCHTRLLIHSMCTSHYLDIFITFIICLNVVTMSLEHYNQPTSLETALKYCNYMFTTVFVLEAVLKLVAFGLRRFFKDRWNQLDLAIVLLSVMGITLEEIEINAALPINPTIIRIMRVLRIARVLKLLKMATGMRALLDTVVQALPQVGNLGLLFMLLFFIYAALGVELFGKLVCNDENPCEGMSRHATFENFGMAFLTLFQVSTGDNWNGIMKDTLRDCTHDERSCLSSLQFVSPLYFVSFVLTAQFVLINVVVAVLMKHLDDSNKEAQEDAEMDAELELEMAHGLGPGPRLPTGSPGAPGRGPGGAGGGGDTEGGLCRRCYSPAQENLWLDSVSLIIKDSLEGELTIIDNLSGSIFHHYSSPAGCKKCHHDKQEVQLAETEAFSLNSDRSSSILLGDDLSLEDPTACPPGRKDSKGELDPPEPMRVGDLGECFFPLSSTAVSPDPENFLCEMEEIPFNPVRSWLKHDSSQAPPSPFSPDASSPLLPMPAEFFHPAVSASQKGPEKGTGTGTLPKIALQGSWASLRSPRVNCTLLRQATGSDTSLDASPSSSAGSLQTTLEDSLTLSDSPRRALGPPAPAPGPRAGLSPAARRRLSLRGRGLFSLRGLRAHQRSHSSGGSTSPGCTHHDSMDPSDEEGRGGAGGGGAGSEHSETLSSLSLTSLFCPPPPPPAPGLTPARKFSSTSSLAAPGRPHAAALAHGLARSPSWAADRSKDPPGRAPLPMGLGPLAPPPQPLPGELEPGDAASKRKR",
# CAC1S_HUMAN
"MEPSSPQDEGLRKKQPKKPVPEILPRPPRALFCLTLENPLRKACISIVEWKPFETIILLTIFANCVALAVYLPMPEDDNNSLNLGLEKLEYFFLIVFSIEAAMKIIAYGFLFHQDAYLRSGWNVLDFTIVFLGVFTVILEQVNVIQSHTAPMSSKGAGLDVKALRAFRVLRPLRLVSGVPSLQVVLNSIFKAMLPLFHIALLVLFMVIIYAIIGLELFKGKMHKTCYFIGTDIVATVENEEPSPCARTGSGRRCTINGSECRGGWPGPNHGITHFDNFGFSMLTVYQCITMEGWTDVLYWVNDAIGNEWPWIYFVTLILLGSFFILNLVLGVLSGEFTKEREKAKSRGTFQKLREKQQLDEDLRGYMSWITQGEVMDVEDFREGKLSLDEGGSDTESLYEIAGLNKIIQFIRHWRQWNRIFRWKCHDIVKSKVFYWLVILIVALNTLSIASEHHNQPLWLTRLQDIANRVLLSLFTTEMLMKMYGLGLRQYFMSIFNRFDCFVVCSGILEILLVESGAMTPLGISVLRCIRLLRIFKITKYWTSLSNLVASLLNSIRSIASLLLLLFLFIVIFALLGMQLFGGRYDFEDTEVRRSNFDNFPQALISVFQVLTGEDWTSMMYNGIMAYGGPSYPGMLVCIYFIILFVCGNYILLNVFLAIAVDNLAEAESLTSAQKAKAEEKKRRKMSKGLPDKSEEEKSTMAKKLEQKPKGEGIPTTAKLKIDEFESNVNEVKDPYPSADFPGDDEEDEPEIPLSPRPRPLAELQLKEKAVPIPEASSFFIFSPTNKIRVLCHRIVNATWFTNFILLFILLSSAALAAEDPIRADSMRNQILKHFDIGFTSVFTVEIVLKMTTYGAFLHKGSFCRNYFNMLDLLVVAVSLISMGLESSAISVVKILRVLRVLRPLRAINRAKGLKHVVQCMFVAISTIGNIVLVTTLLQFMFACIGVQLFKGKFFRCTDLSKMTEEECRGYYYVYKDGDPMQIELRHREWVHSDFHFDNVLSAMMSLFTVSTFEGWPQLLYKAIDSNAEDVGPIYNNRVEMAIFFIIYIILIAFFMMNIFVGFVIVTFQEQGETEYKNCELDKNQRQCVQYALKARPLRCYIPKNPYQYQVWYIVTSSYFEYLMFALIMLNTICLGMQHYNQSEQMNHISDILNVAFTIIFTLEMILKLMAFKARGYFGDPWNVFDFLIVIGSIIDVILSEIDTFLASSGGLYCLGGGCGNVDPDESARISSAFFRLFRVMRLIKLLSRAEGVRTLLWTFIKSFQALPYVALLIVMLFFIYAVIGMQMFGKIALVDGTQINRNNNFQTFPQAVLLLFRCATGEAWQEILLACSYGKLCDPESDYAPGEEYTCGTNFAYYYFISFYMLCAFLVINLFVAVIMDNFDYLTRDWSILGPHHLDEFKAIWAEYDPEAKGRIKHLDVVTLLRRIQPPLGFGKFCPHRVACKRLVGMNMPLNSDGTVTFNATLFALVRTALKIKTEGNFEQANEELRAIIKKIWKRTSMKLLDQVIPPIGDDEVTVGKFYATFLIQEHFRKFMKRQEEYYGYRPKKDIVQIQAGLRTIEEEAAPEICRTVSGDLAAEEELERAMVEAAMEEGIFRRTGGLFGQVDNFLERTNSLPPVMANQRPLQFAEIEMEEMESPVFLEDFPQDPRTNPLARANTNNANANVAYGNSNHSNSHVFSSVHYEREFPEETETPATRGRALGQPCRVLGPHSKPCVEMLKGLLTQRAMPRGQAPPAPCQCPRVESSMPEDRKSSTPGSLHEETPHSRSTRENTSRCSAPATALLIQKALVRGGLGTLAADANFIMATGQALADACQMEPEEVEIMATELLKGREAPEGMASSLGCLNLGSSLGSLDQHQGSQETLIPPRL",
# CA2D1_HUMAN
"MAAGCLLALTLTLFQSLLIGPSSEEPFPSAVTIKSWVDKMQEDLVTLAKTASGVNQLVDIYEKYQDLYTVEPNNARQLVEIAARDIEKLLSNRSKALVRLALEAEKVQAAHQWREDFASNEVVYYNAKDDLDPEKNDSEPGSQRIKPVFIEDANFGRQISYQHAAVHIPTDIYEGSTIVLNELNWTSALDEVFKKNREEDPSLLWQVFGSATGLARYYPASPWVDNSRTPNKIDLYDVRRRPWYIQGAASPKDMLILVDVSGSVSGLTLKLIRTSVSEMLETLSDDDFVNVASFNSNAQDVSCFQHLVQANVRNKKVLKDAVNNITAKGITDYKKGFSFAFEQLLNYNVSRANCNKIIMLFTDGGEERAQEIFNKYNKDKKVRVFTFSVGQHNYDRGPIQWMACENKGYYYEIPSIGAIRINTQEYLDVLGRPMVLAGDKAKQVQWTNVYLDALELGLVITGTLPVFNITGQFENKTNLKNQLILGVMGVDVSLEDIKRLTPRFTLCPNGYYFAIDPNGYVLLHPNLQPKPIGVGIPTINLRKRRPNIQNPKSQEPVTLDFLDAELENDIKVEIRNKMIDGESGEKTFRTLVKSQDERYIDKGNRTYTWTPVNGTDYSLALVLPTYSFYYIKAKLEETITQARYSETLKPDNFEESGYTFIAPRDYCNDLKISDNNTEFLLNFNEFIDRKTPNNPSCNADLINRVLLDAGFTNELVQNYWSKQKNIKGVKARFVVTDGGITRVYPKEAGENWQENPETYEDSFYKRSLDNDNYVFTAPYFNKSGPGAYESGIMVSKAVEIYIQGKLLKPAVVGIKIDVNSWIENFTKTSIRDPCAGPVCDCKRNSDVMDCVILDDGGFLLMANHDDYTNQIGRFFGEIDPSLMRHLVNISVYAFNKSYDYQSVCEPGAAPKQGAGHRSAYVPSVADILQIGWWATAAAWSILQQFLLSLTFPRLLEAVEMEDDDFTASLSKQSCITEQTQYFFDNDSKSFSGVLDCGNCSRIFHGEKLMNTNLIFIMVESKGTCPCDTRLLIQAEQTSDGPNPCDMVKQPRYRKGPDVCFDNNVLEDYTDCGGVSGLNPSLWYIIGIQFLLLWLVSGSTHRLL",
# CA2D2_HUMAN
"MAVPARTCGASRPGPARTARPWPGCGPHPGPGTRRPTSGPPRPLWLLLPLLPLLAAPGASAYSFPQQHTMQHWARRLEQEVDGVMRIFGGVQQLREIYKDNRNLFEVQENEPQKLVEKVAGDIESLLDRKVQALKRLADAAENFQKAHRWQDNIKEEDIVYYDAKADAELDDPESEDVERGSKASTLRLDFIEDPNFKNKVNYSYAAVQIPTDIYKGSTVILNELNWTEALENVFMENRRQDPTLLWQVFGSATGVTRYYPATPWRAPKKIDLYDVRRRPWYIQGASSPKDMVIIVDVSGSVSGLTLKLMKTSVCEMLDTLSDDDYVNVASFNEKAQPVSCFTHLVQANVRNKKVFKEAVQGMVAKGTTGYKAGFEYAFDQLQNSNITRANCNKMIMMFTDGGEDRVQDVFEKYNWPNRTVRVFTFSVGQHNYDVTPLQWMACANKGYYFEIPSIGAIRINTQEYLDVLGRPMVLAGKEAKQVQWTNVYEDALGLGLVVTGTLPVFNLTQDGPGEKKNQLILGVMGIDVALNDIKRLTPNYTLGANGYVFAIDLNGYVLLHPNLKPQTTNFREPVTLDFLDAELEDENKEEIRRSMIDGNKGHKQIRTLVKSLDERYIDEVTRNYTWVPIRSTNYSLGLVLPPYSTFYLQANLSDQILQVKLPISKLKDFEFLLPSSFESEGHVFIAPREYCKDLNASDNNTEFLKNFIELMEKVTPDSKQCNNFLLHNLILDTGITQQLVERVWRDQDLNTYSLLAVFAATDGGITRVFPNKAAEDWTENPEPFNASFYRRSLDNHGYVFKPPHQDALLRPLELENDTVGILVSTAVELSLGRRTLRPAVVGVKLDLEAWAEKFKVLASNRTHQDQPQKCGPNSHCEMDCEVNNEDLLCVLIDDGGFLVLSNQNHQWDQVGRFFSEVDANLMLALYNNSFYTRKESYDYQAACAPQPPGNLGAAPRGVFVPTVADFLNLAWWTSAAAWSLFQQLLYGLIYHSWFQADPAEAEGSPETRESSCVMKQTQYYFGSVNASYNAIIDCGNCSRLFHAQRLTNTNLLFVVAEKPLCSQCEAGRLLQKETHSDGPEQCELVQRPRYRRGPHICFDYNATEDTSDCGRGASFPPSLGVLVSLQLLLLLGLPPRPQPQVLVHASRRL",
# CA2D3_HUMAN
"MAGPGSPRRASRGASALLAAALLYAALGDVVRSEQQIPLSVVKLWASAFGGEIKSIAAKYSGSQLLQKKYKEYEKDVAIEEIDGLQLVKKLAKNMEEMFHKKSEAVRRLVEAAEEAHLKHEFDADLQYEYFNAVLINERDKDGNFLELGKEFILAPNDHFNNLPVNISLSDVQVPTNMYNKDPAIVNGVYWSESLNKVFVDNFDRDPSLIWQYFGSAKGFFRQYPGIKWEPDENGVIAFDCRNRKWYIQAATSPKDVVILVDVSGSMKGLRLTIAKQTVSSILDTLGDDDFFNIIAYNEELHYVEPCLNGTLVQADRTNKEHFREHLDKLFAKGIGMLDIALNEAFNILSDFNHTGQGSICSQAIMLITDGAVDTYDTIFAKYNWPDRKVRIFTYLIGREAAFADNLKWMACANKGFFTQISTLADVQENVMEYLHVLSRPKVIDQEHDVVWTEAYIDSTLPQAQKLTDDQGPVLMTTVAMPVFSKQNETRSKGILLGVVGTDVPVKELLKTIPKYKLGIHGYAFAITNNGYILTHPELRLLYEEGKKRRKPNYSSVDLSEVEWEDRDDVLRNAMVNRKTGKFSMEVKKTVDKGKRVLVMTNDYYYTDIKGTPFSLGVALSRGHGKYFFRGNVTIEEGLHDLEHPDVSLADEWSYCNTDLHPEHRHLSQLEAIKLYLKGKEPLLQCDKELIQEVLFDAVVSAPIEAYWTSLALNKSENSDKGVEVAFLGTRTGLSRINLFVGAEQLTNQDFLKAGDKENIFNADHFPLWYRRAAEQIPGSFVYSIPFSTGPVNKSNVVTASTSIQLLDERKSPVVAAVGIQMKLEFFQRKFWTASRQCASLDGKCSISCDDETVNCYLIDNNGFILVSEDYTQTGDFFGEIEGAVMNKLLTMGSFKRITLYDYQAMCRANKESSDGAHGLLDPYNAFLSAVKWIMTELVLFLVEFNLCSWWHSDMTAKAQKLKQTLEPCDTEYPAFVSERTIKETTGNIACEDCSKSFVIQQIPSSNLFMVVVDSSCLCESVAPITMAPIEIRYNESLKCERLKAQKIRRRPESCHGFHPEENARECGGAPSLQAQTVLLLLPLLLMLFSR",
# CA2D4_HUMAN
"MVCGCSALLPLPNPRPTMPATPNFLANPSSSSRWIPLQPMPVAWAFVQKTSALLWLLLLGTSLSPAWGQAKIPLETVKLWADTFGGDLYNTVTKYSGSLLLQKKYKDVESSLKIEEVDGLELVRKFSEDMENMLRRKVEAVQNLVEAAEEADLNHEFNESLVFDYYNSVLINERDEKGNFVELGAEFLLESNAHFSNLPVNTSISSVQLPTNVYNKDPDILNGVYMSEALNAVFVENFQRDPTLTWQYFGSATGFFRIYPGIKWTPDENGVITFDCRNRGWYIQAATSPKDIVILVDVSGSMKGLRMTIAKHTITTILDTLGENDFINIIAYNDYVHYIEPCFKGILVQADRDNREHFKLLVEELMVKGVGVVDQALREAFQILKQFQEAKQGSLCNQAIMLISDGAVEDYEPVFEKYNWPDCKVRVFTYLIGREVSFADRMKWIACNNKGYYTQISTLADTQENVMEYLHVLSRPMVINHDHDIIWTEAYMDSKLLSSQAQSLTLLTTVAMPVFSKKNETRSHGILLGVVGSDVALRELMKLAPRYKLGVHGYAFLNTNNGYILSHPDLRPLYREGKKLKPKPNYNSVDLSEVEWEDQAESLRTAMINRETGTLSMDVKVPMDKGKRVLFLTNDYFFTDISDTPFSLGVVLSRGHGEYILLGNTSVEEGLHDLLHPDLALAGDWIYCITDIDPDHRKLSQLEAMIRFLTRKDPDLECDEELVREVLFDAVVTAPMEAYWTALALNMSEESEHVVDMAFLGTRAGLLRSSLFVGSEKVSDRKFLTPEDEASVFTLDRFPLWYRQASEHPAGSFVFNLRWAEGPESAGEPMVVTASTAVAVTVDKRTAIAAAAGVQMKLEFLQRKFWAATRQCSTVDGPCTQSCEDSDLDCFVIDNNGFILISKRSRETGRFLGEVDGAVLTQLLSMGVFSQVTMYDYQAMCKPSSHHHSAAQPLVSPISAFLTATRWLLQELVLFLLEWSVWGSWYDRGAEAKSVFHHSHKHKKQDPLQPCDTEYPVFVYQPAIREANGIVECGPCQKVFVVQQIPNSNLLLLVTDPTCDCSIFPPVLQEATEVKYNASVKCDRMRSQKLRRRPDSCHAFHPEENAQDCGGASDTSASPPLLLLPVCAWGLLPQLLR",
# CACB1_HUMAN
"MVQKTSMSRGPYPPSQEIPMEVFDPSPQGKYSKRKGRFKRSDGSTSSDTTSNSFVRQGSAESYTSRPSDSDVSLEEDREALRKEAERQALAQLEKAKTKPVAFAVRTNVGYNPSPGDEVPVQGVAITFEPKDFLHIKEKYNNDWWIGRLVKEGCEVGFIPSPVKLDSLRLLQEQKLRQNRLGSSKSGDNSSSSLGDVVTGTRRPTPPASAKQKQKSTEHVPPYDVVPSMRPIILVGPSLKGYEVTDMMQKALFDFLKHRFDGRISITRVTADISLAKRSVLNNPSKHIIIERSNTRSSLAEVQSEIERIFELARTLQLVALDADTINHPAQLSKTSLAPIIVYIKITSPKVLQRLIKSRGKSQSKHLNVQIAASEKLAQCPPEMFDIILDENQLEDACEHLAEYLEAYWKATHPPSSTPPNPLLNRTMATAALAASPAPVSNLQGPYLASGDQPLERATGEHASMHEYPGELGQPPGLYPSSHPPGRAGTLRALSRQDTFDADTPGSRNSAYTELGDSCVDMETDPSEGPGLGDPAGGGTPPARQGSWEDEEEDYEEELTDNRNRGRNKARYCAEGGGPVLGRNKNELEGWGRGVYIR",
# CACB2_HUMAN
"MVQRDMSKSPPTAAAAVAQEIQMELLENVAPAGALGAAAQSYGKGARRKNRFKGSDGSTSSDTTSNSFVRQGSADSYTSRPSDSDVSLEEDREAVRREAERQAQAQLEKAKTKPVAFAVRTNVSYSAAHEDDVPVPGMAISFEAKDFLHVKEKFNNDWWIGRLVKEGCEIGFIPSPVKLENMRLQHEQRAKQGKFYSSKSGGNSSSSLGDIVPSSRKSTPPSSAIDIDATGLDAEENDIPANHRSPKPSANSVTSPHSKEKRMPFFKKTEHTPPYDVVPSMRPVVLVGPSLKGYEVTDMMQKALFDFLKHRFEGRISITRVTADISLAKRSVLNNPSKHAIIERSNTRSSLAEVQSEIERIFELARTLQLVVLDADTINHPAQLSKTSLAPIIVYVKISSPKVLQRLIKSRGKSQAKHLNVQMVAADKLAQCPPELFDVILDENQLEDACEHLADYLEAYWKATHPPSSSLPNPLLSRTLATSSLPLSPTLASNSQGSQGDQRTDRSAPIRSASQAEEEPSVEPVKKSQHRSSSSAPHHNHRSGTSRGLSRQETFDSETQESRDSAYVEPKEDYSHDHVDHYASHRDHNHRDETHGSSDHRHRESRHRSRDVDREQDHNECNKQRSRHKSKDRYCEKDGEVISKKRNEAGEWNRDVYIRQ",
# CACB3_HUMAN
"MYDDSYVPGFEDSEAGSADSYTSRPSLDSDVSLEEDRESARREVESQAQQQLERAKHKPVAFAVRTNVSYCGVLDEECPVQGSGVNFEAKDFLHIKEKYSNDWWIGRLVKEGGDIAFIPSPQRLESIRLKQEQKARRSGNPSSLSDIGNRRSPPPSLAKQKQKQAEHVPPYDVVPSMRPVVLVGPSLKGYEVTDMMQKALFDFLKHRFDGRISITRVTADLSLAKRSVLNNPGKRTIIERSSARSSIAEVQSEIERIFELAKSLQLVVLDADTINHPAQLAKTSLAPIIVFVKVSSPKVLQRLIRSRGKSQMKHLTVQMMAYDKLVQCPPESFDVILDENQLEDACEHLAEYLEVYWRATHHPAPGPGLLGPPSAIPGLQNQQLLGERGEEHSPLERDSLMPSDEASESSRQAWTGSSQRSSRHLEEDYADAYQDLYQPHRQHTSGLPSANGHDPQDRLLAQDSEHNHSDRNWQRNRPWPKDSY",
# CACB4_HUMAN
"MSSSSYAKNGTADGPHSPTSQVARGTTTRRSRLKRSDGSTTSTSFILRQGSADSYTSRPSDSDVSLEEDREAIRQEREQQAAIQLERAKSKPVAFAVKTNVSYCGALDEDVPVPSTAISFDAKDFLHIKEKYNNDWWIGRLVKEGCEIGFIPSPLRLENIRIQQEQKRGRFHGGKSSGNSSSSLGEMVSGTFRATPTSTAKQKQKVTEHIPPYDVVPSMRPVVLVGPSLKGYEVTDMMQKALFDFLKHRFDGRISITRVTADISLAKRSVLNNPSKRAIIERSNTRSSLAEVQSEIERIFELARSLQLVVLDADTINHPAQLIKTSLAPIIVHVKVSSPKVLQRLIKSRGKSQSKHLNVQLVAADKLAQCPPEMFDVILDENQLEDACEHLGEYLEAYWRATHTTSSTPMTPLLGRNLGSTALSPYPTAISGLQSQRMRHSNHSTENSPIERRSLMTSDENYHNERARKSRNRLSSSSQHSRDHYPLVEEDYPDSYQDTYKPHRNRGSPGGYSHDSRHRL",
# CTSR1_HUMAN
"MDQNSVPEKAQNEADTNNADRFFRSHSSPPHHRPGHSRALHHYELHHHGVPHQRGESHHPPEFQDFHDQALSSHVHQSHHHSEARNHGRAHGPTGFGLAPSQGAVPSHRSYGEDYHDELQRDGRRHHDGSQYGGFHQQSDSHYHRGSHHGRPQYLGENLSHYSSGVPHHGEASHHGGSYLPHGPNPYSESFHHSEASHLSGLQHDESQHHQVPHRGWPHHHQVHHHGRSRHHEAHQHGKSPHHGETISPHSSVGSYQRGISDYHSEYHQGDHHPSEYHHGDHPHHTQHHYHQTHRHRDYHQHQDHHGAYHSSYLHGDYVQSTSQLSIPHTSRSLIHDAPGPAASRTGVFPYHVAHPRGSAHSMTRSSSTIRSRVTQMSKKVHTQDISTKHSEDWGKEEGQFQKRKTGRLQRTRKKGHSTNLFQWLWEKLTFLIQGFREMIRNLTQSLAFETFIFFVVCLNTVMLVAQTFAEVEIRGEWYFMALDSIFFCIYVVEALLKIIALGLSYFFDFWNNLDFFIMAMAVLDFLLMQTHSFAIYHQSLFRILKVFKSLRALRAIRVLRRLSFLTSVQEVTGTLGQSLPSIAAILILMFTCLFLFSAVLRALFRKSDPKRFQNIFTTIFTLFTLLTLDDWSLIYMDSRAQGAWYIIPILVIYIIIQYFIFLNLVITVLVDSFQTALFKGLEKAKQERAARIQEKLLEDSLTELRAAEPKEVASEGTMLKRLIEKKFGTMTEKQQELLFHYLQLVASVEQEQQKFRSQAAVIDEIVDTTFEAGEEDFRN",
# CTSR2_HUMAN
"MAAYQQEEQMQLPRADAIRSRLIDTFSLIEHLQGLSQAVPRHTIRELLDPSRQKKLVLGDQHQLVRFSIKPQRIEQISHAQRLLSRLHVRCSQRPPLSLWAGWVLECPLFKNFIIFLVFLNTIILMVEIELLESTNTKLWPLKLTLEVAAWFILLIFILEILLKWLSNFSVFWKSAWNVFDFVVTMLSLLPEVVVLVGVTGQSVWLQLLRICRVLRSLKLLAQFRQIQIIILVLVRALKSMTFLLMLLLIFFYIFAVTGVYVFSEYTRSPRQDLEYHVFFSDLPNSLVTVFILFTLDHWYALLQDVWKVPEVSRIFSSIYFILWLLLGSIIFRSIIVAMMVTNFQNIRKELNEEMARREVQLKADMFKRQIIQRRKNMSHEALTSSHSKIEDSSRGASQQRESLDLSEVSEVESNYGATEEDLITSASKTEETLSKKREYQSSSCVSSTSSSYSSSSESRFSESIGRLDWETLVHENLPGLMEMDQDDRVWPRDSLFRYFELLEKLQYNLEERKKLQEFAVQALMNLEDK",
# CTSR3_HUMAN
"MSQHRHQRHSRVISSSPVDTTSVGFCPTFKKFKRNDDECRAFVKRVIMSRFFKIIMISTVTSNAFFMALWTSYDIRYRLFRLLEFSEIFFVSICTSELSMKVYVDPINYWKNGYNLLDVIIIIVMFLPYALRQLMGKQFTYLYIADGMQSLRILKLIGYSQGIRTLITAVGQTVYTVASVLLLLFLLMYIFAILGFCLFGSPDNGDHDNWGNLAAAFFTLFSLATVDGWTDLQKQLDNREFALSRAFTIIFILLASFIFLNMFVGVMIMHTEDSIRKFERELMLEQQEMLMGEKQVILQRQQEEISRLMHIQKNADCTSFSELVENFKKTLSHTDPMVLDDFGTSLPFIDIYFSTLDYQDTTVHKLQELYYEIVHVLSLMLEDLPQEKPQSLEKVDEK",
# CTSR4_HUMAN
"MRDNEKAWWQQWTSHTGLEGWGGTQEDRMGFGGAVAALRGRPSPLQSTIHESYGRPEEQVLINRQEITNKADAWDMQEFITHMYIKQLLRHPAFQLLLALLLVINAITIALRTNSYLDQKHYELFSTIDDIVLTILLCEVLLGWLNGFWIFWKDGWNILNFIIVFILLLRFFINEINIPSINYTLRALRLVHVCMAVEPLARIIRVILQSVPDMANIMVLILFFMLVFSVFGVTLFGAFVPKHFQNIQVALYTLFICITQDGWVDIYSDFQTEKREYAMEIGGAIYFTIFITIGAFIGINLFVIVVTTNLEQMMKAGEQGQQQRITFSETGAEEEEENDQLPLVHCVVARSEKSGLLQEPLAGGPLSNLSENTCDNFCLVLEAIQENLRQYKEIRDELNMIVEEVRAIRFNQEQESEVLNRRSSTSGSLETTSSKDIRQMSQQQDLLSALVSMEKVHDSSSQILLKKHKSSH",
# ITPR1_HUMAN
"MSDKMSSFLHIGDICSLYAEGSTNGFISTLGLVDDRCVVQPETGDLNNPPKKFRDCLFKLCPMNRYSAQKQFWKAAKPGANSTTDAVLLNKLHHAADLEKKQNETENRKLLGTVIQYGNVIQLLHLKSNKYLTVNKRLPALLEKNAMRVTLDEAGNEGSWFYIQPFYKLRSIGDSVVIGDKVVLNPVNAGQPLHASSHQLVDNPGCNEVNSVNCNTSWKIVLFMKWSDNKDDILKGGDVVRLFHAEQEKFLTCDEHRKKQHVFLRTTGRQSATSATSSKALWEVEVVQHDPCRGGAGYWNSLFRFKHLATGHYLAAEVDPDFEEECLEFQPSVDPDQDASRSRLRNAQEKMVYSLVSVPEGNDISSIFELDPTTLRGGDSLVPRNSYVRLRHLCTNTWVHSTNIPIDKEEEKPVMLKIGTSPVKEDKEAFAIVPVSPAEVRDLDFANDASKVLGSIAGKLEKGTITQNERRSVTKLLEDLVYFVTGGTNSGQDVLEVVFSKPNRERQKLMREQNILKQIFKLLQAPFTDCGDGPMLRLEELGDQRHAPFRHICRLCYRVLRHSQQDYRKNQEYIAKQFGFMQKQIGYDVLAEDTITALLHNNRKLLEKHITAAEIDTFVSLVRKNREPRFLDYLSDLCVSMNKSIPVTQELICKAVLNPTNADILIETKLVLSRFEFEGVSSTGENALEAGEDEEEVWLFWRDSNKEIRSKSVRELAQDAKEGQKEDRDVLSYYRYQLNLFARMCLDRQYLAINEISGQLDVDLILRCMSDENLPYDLRASFCRLMLHMHVDRDPQEQVTPVKYARLWSEIPSEIAIDDYDSSGASKDEIKERFAQTMEFVEEYLRDVVCQRFPFSDKEKNKLTFEVVNLARNLIYFGFYNFSDLLRLTKILLAILDCVHVTTIFPISKMAKGEENKGNNDVEKLKSSNVMRSIHGVGELMTQVVLRGGGFLPMTPMAAAPEGNVKQAEPEKEDIMVMDTKLKIIEILQFILNVRLDYRISCLLCIFKREFDESNSQTSETSSGNSSQEGPSNVPGALDFEHIEEQAEGIFGGSEENTPLDLDDHGGRTFLRVLLHLTMHDYPPLVSGALQLLFRHFSQRQEVLQAFKQVQLLVTSQDVDNYKQIKQDLDQLRSIVEKSELWVYKGQGPDETMDGASGENEHKKTEEGNNKPQKHESTSSYNYRVVKEILIRLSKLCVQESASVRKSRKQQQRLLRNMGAHAVVLELLQIPYEKAEDTKMQEIMRLAHEFLQNFCAGNQQNQALLHKHINLFLNPGILEAVTMQHIFMNNFQLCSEINERVVQHFVHCIETHGRNVQYIKFLQTIVKAEGKFIKKCQDMVMAELVNSGEDVLVFYNDRASFQTLIQMMRSERDRMDENSPLMYHIHLVELLAVCTEGKNVYTEIKCNSLLPLDDIVRVVTHEDCIPEVKIAYINFLNHCYVDTEVEMKEIYTSNHMWKLFENFLVDICRACNNTSDRKHADSILEKYVTEIVMSIVTTFFSSPFSDQSTTLQTRQPVFVQLLQGVFRVYHCNWLMPSQKASVESCIRVLSDVAKSRAIAIPVDLDSQVNNLFLKSHSIVQKTAMNWRLSARNAARRDSVLAASRDYRNIIERLQDIVSALEDRLRPLVQAELSVLVDVLHRPELLFPENTDARRKCESGGFICKLIKHTKQLLEENEEKLCIKVLQTLREMMTKDRGYGEKLISIDELDNAELPPAPDSENATEELEPSPPLRQLEDHKRGEALRQVLVNRYYGNVRPSGRRESLTSFGNGPLSAGGPGKPGGGGGGSGSSSMSRGEMSLAEVQCHLDKEGASNLVIDLIMNASSDRVFHESILLAIALLEGGNTTIQHSFFCRLTEDKKSEKFFKVFYDRMKVAQQEIKATVTVNTSDLGNKKKDDEVDRDAPSRKKAKEPTTQITEEVRDQLLEASAATRKAFTTFRREADPDDHYQPGEGTQATADKAKDDLEMSAVITIMQPILRFLQLLCENHNRDLQNFLRCQNNKTNYNLVCETLQFLDCICGSTTGGLGLLGLYINEKNVALINQTLESLTEYCQGPCHENQNCIATHESNGIDIITALILNDINPLGKKRMDLVLELKNNASKLLLAIMESRHDSENAERILYNMRPKELVEVIKKAYMQGEVEFEDGENGEDGAASPRNVGHNIYILAHQLARHNKELQSMLKPGGQVDGDEALEFYAKHTAQIEIVRLDRTMEQIVFPVPSICEFLTKESKLRIYYTTERDEQGSKINDFFLRSEDLFNEMNWQKKLRAQPVLYWCARNMSFWSSISFNLAVLMNLLVAFFYPFKGVRGGTLEPHWSGLLWTAMLISLAIVIALPKPHGIRALIASTILRLIFSVGLQPTLFLLGAFNVCNKIIFLMSFVGNCGTFTRGYRAMVLDVEFLYHLLYLVICAMGLFVHEFFYSLLLFDLVYREETLLNVIKSVTRNGRSIILTAVLALILVYLFSIVGYLFFKDDFILEVDRLPNETAVPETGESLASEFLFSDVCRVESGENCSSPAPREELVPAEETEQDKEHTCETLLMCIVTVLSHGLRSGGGVGDVLRKPSKEEPLFAARVIYDLLFFFMVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVTFEEHIKEEHNMWHYLCFIVLVKVKDSTEYTGPESYVAEMIKERNLDWFPRMRAMSLVSSDSEGEQNELRNLQEKLESTMKLVTNLSGQLSELKDQMTEQRKQKQRIGLLGHPPHMNVNPQQPA",
# ITPR2_HUMAN
"MTEKMSSFLYIGDIVSLYAEGSVNGFISTLGLVDDRCVVHPEAGDLANPPKKFRDCLFKVCPMNRYSAQKQYWKAKQAKQGNHTEAALLKKLQHAAELEQKQNESENKKLLGEIVKYSNVIQLLHIKSNKYLTVNKRLPALLEKNAMRVSLDAAGNEGSWFYIHPFWKLRSEGDNIVVGDKVVLMPVNAGQPLHASNIELLDNPGCKEVNAVNCNTSWKITLFMKYSSYREDVLKGGDVVRLFHAEQEKFLTCDEYEKKQHIFLRTTLRQSATSATSSKALWEIEVVHHDPCRGGAGQWNSLFRFKHLATGNYLAAELNPDYRDAQNEGKNVRDGVPPTSKKKRQAGEKIMYTLVSVPHGNDIASLFELDATTLQRADCLVPRNSYVRLRHLCTNTWVTSTSIPIDTDEERPVMLKIGTCQTKEDKEAFAIVSVPLSEVRDLDFANDANKVLATTVKKLENGTITQNERRFVTKLLEDLIFFVADVPNNGQEVLDVVITKPNRERQKLMREQNILAQVFGILKAPFKEKAGEGSMLRLEDLGDQRYAPYKYMLRLCYRVLRHSQQDYRKNQEYIAKNFCVMQSQIGYDILAEDTITALLHNNRKLLEKHITAKEIETFVSLLRRNREPRFLDYLSDLCVSNTTAIPVTQELICKFMLSPGNADILIQTKVVSMQADNPMESSILSDDIDDEEVWLYWIDSNKEPHGKAIRHLAQEAKEGTKADLEVLTYYRYQLNLFARMCLDRQYLAINQISTQLSVDLILRCVSDESLPFDLRASFCRLMLHMHVDRDPQESVVPVRYARLWTEIPTKITIHEYDSITDSSRNDMKRKFALTMEFVEEYLKEVVNQPFPFGDKEKNKLTFEVVHLARNLIYFGFYSFSELLRLTRTLLAILDIVQAPMSSYFERLSKFQDGGNNVMRTIHGVGEMMTQMVLSRGSIFPMSVPDVPPSIHPSKQGSPTEHEDVTVMDTKLKIIEILQFILSVRLDYRISYMLSIYKKEFGEDNDNAETSASGSPDTLLPSAIVPDIDEIAAQAETMFAGRKEKNPVQLDDEGGRTFLRVLIHLIMHDYPPLLSGALQLLFKHFSQRAEVLQAFKQVQLLVSNQDVDNYKQIKADLDQLRLTVEKSELWVEKSSNYENGEIGESQVKGGEEPIEESNILSPVQDGTKKPQIDSNKSNNYRIVKEILIRLSKLCVQNKKCRNQHQRLLKNMGAHSVVLDLLQIPYEKNDEKMNEVMNLAHTFLQNFCRGNPQNQVLLHKHLNLFLTPGLLEAETMRHIFMNNYHLCNEISERVVQHFVHCIETHGRHVEYLRFLQTIVKADGKYVKKCQDMVMTELINGGEDVLIFYNDRASFPILLHMMCSERDRGDESGPLAYHITLVELLAACTEGKNVYTEIKCNSLLPLDDIVRVVTHDDCIPEVKIAYVNFVNHCYVDTEVEMKEIYTSNHIWKLFENFLVDMARVCNTTTDRKHADIFLEKCVTESIMNIVSGFFNSPFSDNSTSLQTHQPVFIQLLQSAFRIYNCTWPNPAQKASVESCIRTLAEVAKNRGIAIPVDLDSQVNTLFMKSHSNMVQRAAMGWRLSARSGPRFKEALGGPAWDYRNIIEKLQDVVASLEHQFSPMMQAEFSVLVDVLYSPELLFPEGSDARIRCGAFMSKLINHTKKLMEKEEKLCIKILQTLREMLEKKDSFVEEGNTLRKILLNRYFKGDYSIGVNGHLSGAYSKTAQVGGSFSGQDSDKMGISMSDIQCLLDKEGASELVIDVIVNTKNDRIFSEGIFLGIALLEGGNTQTQYSFYQQLHEQKKSEKFFKVLYDRMKAAQKEIRSTVTVNTIDLGNKKRDDDNELMTSGPRMRVRDSTLHLKEGMKGQLTEASSATSKAYCVYRREMDPEIDIMCTGPEAGNTEEKSAEEVTMSPAIAIMQPILRFLQLLCENHNRELQNFLRNQNNKTNYNLVCETLQFLDCICGSTTGGLGLLGLYINEKNVALVNQNLESLTEYCQGPCHENQTCIATHESNGIDIIIALILNDINPLGKYRMDLVLQLKNNASKLLLAIMESRHDSENAERILFNMRPRELVDVMKNAYNQGLECDHGDDEGGDDGVSPKDVGHNIYILAHQLARHNKLLQQMLKPGSDPDEGDEALKYYANHTAQIEIVRHDRTMEQIVFPVPNICEYLTRESKCRVFNTTERDEQGSKVNDFFQQTEDLYNEMKWQKKIRNNPALFWFSRHISLWGSISFNLAVFINLAVALFYPFGDDGDEGTLSPLFSVLLWIAVAICTSMLFFFSKPVGIRPFLVSIMLRSIYTIGLGPTLILLGAANLCNKIVFLVSFVGNRGTFTRGYRAVILDMAFLYHVAYVLVCMLGLFVHEFFYSFLLFDLVYREETLLNVIKSVTRNGRSIILTAVLALILVYLFSIIGFLFLKDDFTMEVDRLKNRTPVTGSHQVPTMTLTTMMEACAKENCSPTIPASNTADEEYEDGIERTCDTLLMCIVTVLNQGLRNGGGVGDVLRRPSKDEPLFAARVVYDLLFYFIVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVSFEEHIKSEHNMWHYLYFIVLVKVKDPTEYTGPESYVAQMIVEKNLDWFPRMRAMSLVSNEGDSEQNEIRSLQEKLESTMSLVKQLSGQLAELKEQMTEQRKNKQRLGFLGSNTPHVNHHMPPH",
# ITPR3_HUMAN
"MSEMSSFLHIGDIVSLYAEGSVNGFISTLGLVDDRCVVEPAAGDLDNPPKKFRDCLFKVCPMNRYSAQKQYWKAKQTKQDKEKIADVVLLQKLQHAAQMEQKQNDTENKKVHGDVVKYGSVIQLLHMKSNKYLTVNKRLPALLEKNAMRVTLDATGNEGSWLFIQPFWKLRSNGDNVVVGDKVILNPVNAGQPLHASNYELSDNAGCKEVNSVNCNTSWKINLFMQFRDHLEEVLKGGDVVRLFHAEQEKFLTCDEYKGKLQVFLRTTLRQSATSATSSNALWEVEVVHHDPCRGGAGHWNGLYRFKHLATGNYLAAEENPSYKGDASDPKAAGMGAQGRTGRRNAGEKIKYCLVAVPHGNDIASLFELDPTTLQKTDSFVPRNSYVRLRHLCTNTWIQSTNVPIDIEEERPIRLMLGTCPTKEDKEAFAIVSVPVSEIRDLDFANDASSMLASAVEKLNEGFISQNDRRFVIQLLEDLVFFVSDVPNNGQNVLDIMVTKPNRERQKLMREQNILKQVFGILKAPFREKGGEGPLVRLEELSDQKNAPYQHMFRLCYRVLRHSQEDYRKNQEHIAKQFGMMQSQIGYDILAEDTITALLHNNRKLLEKHITKTEVETFVSLVRKNREPRFLDYLSDLCVSNHIAIPVTQELICKCVLDPKNSDILIRTELRPVKEMAQSHEYLSIEYSEEEVWLTWTDKNNEHHEKSVRQLAQEARAGNAHDENVLSYYRYQLKLFARMCLDRQYLAIDEISQQLGVDLIFLCMADEMLPFDLRASFCHLMLHVHVDRDPQELVTPVKFARLWTEIPTAITIKDYDSNLNASRDDKKNKFANTMEFVEDYLNNVVSEAVPFANEEKNKLTFEVVSLAHNLIYFGFYSFSELLRLTRTLLGIIDCVQGPPAMLQAYEDPGGKNVRRSIQGVGHMMSTMVLSRKQSVFSAPSLSAGASAAEPLDRSKFEENEDIVVMETKLKILEILQFILNVRLDYRISYLLSVFKKEFVEVFPMQDSGADGTAPAFDSTTANMNLDRIGEQAEAMFGVGKTSSMLEVDDEGGRMFLRVLIHLTMHDYAPLVSGALQLLFKHFSQRQEAMHTFKQVQLLISAQDVENYKVIKSELDRLRTMVEKSELWVDKKGSGKGEEVEAGAAKDKKERPTDEEGFLHPPGEKSSENYQIVKGILERLNKMCGVGEQMRKKQQRLLKNMDAHKVMLDLLQIPYDKGDAKMMEILRYTHQFLQKFCAGNPGNQALLHKHLHLFLTPGLLEAETMQHIFLNNYQLCSEISEPVLQHFVHLLATHGRHVQYLDFLHTVIKAEGKYVKKCQDMIMTELTNAGDDVVVFYNDKASLAHLLDMMKAARDGVEDHSPLMYHISLVDLLAACAEGKNVYTEIKCTSLLPLEDVVSVVTHEDCITEVKMAYVNFVNHCYVDTEVEMKEIYTSNHIWTLFENFTLDMARVCSKREKRVADPTLEKYVLSVVLDTINAFFSSPFSENSTSLQTHQTIVVQLLQSTTRLLECPWLQQQHKGSVEACIRTLAMVAKGRAILLPMDLDAHISSMLSSGASCAAAAQRNASSYKATTRAFPRVTPTANQWDYKNIIEKLQDIITALEERLKPLVQAELSVLVDVLHWPELLFLEGSEAYQRCESGGFLSKLIQHTKDLMESEEKLCIKVLRTLQQMLLKKTKYGDRGNQLRKMLLQNYLQNRKSTSRGDLPDPIGTGLDPDWSAIAATQCRLDKEGATKLVCDLITSTKNEKIFQESIGLAIHLLDGGNTEIQKSFHNLMMSDKKSERFFKVLHDRMKRAQQETKSTVAVNMNDLGSQPHEDREPVDPTTKGRVASFSIPGSSSRYSLGPSLRRGHEVSERVQSSEMGTSVLIMQPILRFLQLLCENHNRDLQNFLRCQNNKTNYNLVCETLQFLDIMCGSTTGGLGLLGLYINEDNVGLVIQTLETLTEYCQGPCHENQTCIVTHESNGIDIITALILNDISPLCKYRMDLVLQLKDNASKLLLALMESRHDSENAERILISLRPQELVDVIKKAYLQEEERENSEVSPREVGHNIYILALQLSRHNKQLQHLLKPVKRIQEEEAEGISSMLSLNNKQLSQMLKSSAPAQEEEEDPLAYYENHTSQIEIVRQDRSMEQIVFPVPGICQFLTEETKHRLFTTTEQDEQGSKVSDFFDQSSFLHNEMEWQRKLRSMPLIYWFSRRMTLWGSISFNLAVFINIIIAFFYPYMEGASTGVLDSPLISLLFWILICFSIAALFTKRYSIRPLIVALILRSIYYLGIGPTLNILGALNLTNKIVFVVSFVGNRGTFIRGYKAMVMDMEFLYHVGYILTSVLGLFAHELFYSILLFDLIYREETLFNVIKSVTRNGRSILLTALLALILVYLFSIVGFLFLKDDFILEVDRLPNNHSTASPLGMPHGAAAFVDTCSGDKMDCVSGLSVPEVLEEDRELDSTERACDTLLMCIVTVMNHGLRNGGGVGDILRKPSKDESLFPARVVYDLLFFFIVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVSFEEHIKLEHNMWNYLYFIVLVRVKNKTDYTGPESYVAQMIKNKNLDWFPRMRAMSLVSNEGEGEQNEIRILQDKLNSTMKLVSHLTAQLNELKEQMTEQRKRRQRLGFVDVQNCISR",
# RYR1_HUMAN
"MGDAEGEDEVQFLRTDDEVVLQCSATVLKEQLKLCLAAEGFGNRLCFLEPTSNAQNVPPDLAICCFVLEQSLSVRALQEMLANTVEAGVESSQGGGHRTLLYGHAILLRHAHSRMYLSCLTTSRSMTDKLAFDVGLQEDATGEACWWTMHPASKQRSEGEKVRVGDDIILVSVSSERYLHLSTASGELQVDASFMQTLWNMNPICSRCEEGFVTGGHVLRLFHGHMDECLTISPADSDDQRRLVYYEGGAVCTHARSLWRLEPLRISWSGSHLRWGQPLRVRHVTTGQYLALTEDQGLVVVDASKAHTKATSFCFRISKEKLDVAPKRDVEGMGPPEIKYGESLCFVQHVASGLWLTYAAPDPKALRLGVLKKKAMLHQEGHMDDALSLTRCQQEESQAARMIHSTNGLYNQFIKSLDSFSGKPRGSGPPAGTALPIEGVILSLQDLIIYFEPPSEDLQHEEKQSKLRSLRNRQSLFQEEGMLSMVLNCIDRLNVYTTAAHFAEFAGEEAAESWKEIVNLLYELLASLIRGNRSNCALFSTNLDWLVSKLDRLEASSGILEVLYCVLIESPEVLNIIQENHIKSIISLLDKHGRNHKVLDVLCSLCVCNGVAVRSNQDLITENLLPGRELLLQTNLINYVTSIRPNIFVGRAEGTTQYSKWYFEVMVDEVTPFLTAQATHLRVGWALTEGYTPYPGAGEGWGGNGVGDDLYSYGFDGLHLWTGHVARPVTSPGQHLLAPEDVISCCLDLSVPSISFRINGCPVQGVFESFNLDGLFFPVVSFSAGVKVRFLLGGRHGEFKFLPPPGYAPCHEAVLPRERLHLEPIKEYRREGPRGPHLVGPSRCLSHTDFVPCPVDTVQIVLPPHLERIREKLAENIHELWALTRIEQGWTYGPVRDDNKRLHPCLVDFHSLPEPERNYNLQMSGETLKTLLALGCHVGMADEKAEDNLKKTKLPKTYMMSNGYKPAPLDLSHVRLTPAQTTLVDRLAENGHNVWARDRVGQGWSYSAVQDIPARRNPRLVPYRLLDEATKRSNRDSLCQAVRTLLGYGYNIEPPDQEPSQVENQSRCDRVRIFRAEKSYTVQSGRWYFEFEAVTTGEMRVGWARPELRPDVELGADELAYVFNGHRGQRWHLGSEPFGRPWQPGDVVGCMIDLTENTIIFTLNGEVLMSDSGSETAFREIEIGDGFLPVCSLGPGQVGHLNLGQDVSSLRFFAICGLQEGFEPFAINMQRPVTTWFSKGLPQFEPVPLEHPHYEVSRVDGTVDTPPCLRLTHRTWGSQNSLVEMLFLRLSLPVQFHQHFRCTAGATPLAPPGLQPPAEDEARAAEPDPDYENLRRSAGGWSEAENGKEGTAKEGAPGGTPQAGGEAQPARAENEKDATTEKNKKRGFLFKAKKVAMMTQPPATPTLPRLPHDVVPADNRDDPEIILNTTTYYYSVRVFAGQEPSCVWAGWVTPDYHQHDMSFDLSKVRVVTVTMGDEQGNVHSSLKCSNCYMVWGGDFVSPGQQGRISHTDLVIGCLVDLATGLMTFTANGKESNTFFQVEPNTKLFPAVFVLPTHQNVIQFELGKQKNIMPLSAAMFQSERKNPAPQCPPRLEMQMLMPVSWSRMPNHFLQVETRRAGERLGWAVQCQEPLTMMALHIPEENRCMDILELSERLDLQRFHSHTLRLYRAVCALGNNRVAHALCSHVDQAQLLHALEDAHLPGPLRAGYYDLLISIHLESACRSRRSMLSEYIVPLTPETRAITLFPPGRSTENGHPRHGLPGVGVTTSLRPPHHFSPPCFVAALPAAGAAEAPARLSPAIPLEALRDKALRMLGEAVRDGGQHARDPVGGSVEFQFVPVLKLVSTLLVMGIFGDEDVKQILKMIEPEVFTEEEEEEDEEEEGEEEDEEEKEEDEEETAQEKEDEEKEEEEAAEGEKEEGLEEGLLQMKLPESVKLQMCHLLEYFCDQELQHRVESLAAFAERYVDKLQANQRSRYGLLIKAFSMTAAETARRTREFRSPPQEQINMLLQFKDGTDEEDCPLPEEIRQDLLDFHQDLLAHCGIQLDGEEEEPEEETTLGSRLMSLLEKVRLVKKKEEKPEEERSAEESKPRSLQELVSHMVVRWAQEDFVQSPELVRAMFSLLHRQYDGLGELLRALPRAYTISPSSVEDTMSLLECLGQIRSLLIVQMGPQEENLMIQSIGNIMNNKVFYQHPNLMRALGMHETVMEVMVNVLGGGESKEIRFPKMVTSCCRFLCYFCRISRQNQRSMFDHLSYLLENSGIGLGMQGSTPLDVAAASVIDNNELALALQEQDLEKVVSYLAGCGLQSCPMLVAKGYPDIGWNPCGGERYLDFLRFAVFVNGESVEENANVVVRLLIRKPECFGPALRGEGGSGLLAAIEEAIRISEDPARDGPGIRRDRRREHFGEEPPEENRVHLGHAIMSFYAALIDLLGRCAPEMHLIQAGKGEALRIRAILRSLVPLEDLVGIISLPLQIPTLGKDGALVQPKMSASFVPDHKASMVLFLDRVYGIENQDFLLHVLDVGFLPDMRAAASLDTATFSTTEMALALNRYLCLAVLPLITKCAPLFAGTEHRAIMVDSMLHTVYRLSRGRSLTKAQRDVIEDCLMSLCRYIRPSMLQHLLRRLVFDVPILNEFAKMPLKLLTNHYERCWKYYCLPTGWANFGVTSEEELHLTRKLFWGIFDSLAHKKYDPELYRMAMPCLCAIAGALPPDYVDASYSSKAEKKATVDAEGNFDPRPVETLNVIIPEKLDSFINKFAEYTHEKWAFDKIQNNWSYGENIDEELKTHPMLRPYKTFSEKDKEIYRWPIKESLKAMIAWEWTIEKAREGEEEKTEKKKTRKISQSAQTYDPREGYNPQPPDLSAVTLSRELQAMAEQLAENYHNTWGRKKKQELEAKGGGTHPLLVPYDTLTAKEKARDREKAQELLKFLQMNGYAVTRGLKDMELDSSSIEKRFAFGFLQQLLRWMDISQEFIAHLEAVVSSGRVEKSPHEQEIKFFAKILLPLINQYFTNHCLYFLSTPAKVLGSGGHASNKEKEMITSLFCKLAALVRHRVSLFGTDAPAVVNCLHILARSLDARTVMKSGPEIVKAGLRSFFESASEDIEKMVENLRLGKVSQARTQVKGVGQNLTYTTVALLPVLTTLFQHIAQHQFGDDVILDDVQVSCYRTLCSIYSLGTTKNTYVEKLRPALGECLARLAAAMPVAFLEPQLNEYNACSVYTTKSPRERAILGLPNSVEEMCPDIPVLERLMADIGGLAESGARYTEMPHVIEITLPMLCSYLPRWWERGPEAPPSALPAGAPPPCTAVTSDHLNSLLGNILRIIVNNLGIDEASWMKRLAVFAQPIVSRARPELLQSHFIPTIGRLRKRAGKVVSEEEQLRLEAKAEAQEGELLVRDEFSVLCRDLYALYPLLIRYVDNNRAQWLTEPNPSAEELFRMVGEIFIYWSKSHNFKREEQNFVVQNEINNMSFLTADNKSKMAKAGDIQSGGSDQERTKKKRRGDRYSVQTSLIVATLKKMLPIGLNMCAPTDQDLITLAKTRYALKDTDEEVREFLHNNLHLQGKVEGSPSLRWQMALYRGVPGREEDADDPEKIVRRVQEVSAVLYYLDQTEHPYKSKKAVWHKLLSKQRRRAVVACFRMTPLYNLPTHRACNMFLESYKAAWILTEDHSFEDRMIDDLSKAGEQEEEEEEVEEKKPDPLHQLVLHFSRTALTEKSKLDEDYLYMAYADIMAKSCHLEEGGENGEAEEEVEVSFEEKQMEKQRLLYQQARLHTRGAAEMVLQMISACKGETGAMVSSTLKLGISILNGGNAEVQQKMLDYLKDKKEVGFFQSIQALMQTCSVLDLNAFERQNKAEGLGMVNEDGTVINRQNGEKVMADDEFTQDLFRFLQLLCEGHNNDFQNYLRTQTGNTTTINIIICTVDYLLRLQESISDFYWYYSGKDVIEEQGKRNFSKAMSVAKQVFNSLTEYIQGPCTGNQQSLAHSRLWDAVVGFLHVFAHMMMKLAQDSSQIELLKELLDLQKDMVVMLLSLLEGNVVNGMIARQMVDMLVESSSNVEMILKFFDMFLKLKDIVGSEAFQDYVTDPRGLISKKDFQKAMDSQKQFSGPEIQFLLSCSEADENEMINCEEFANRFQEPARDIGFNVAVLLTNLSEHVPHDPRLHNFLELAESILEYFRPYLGRIEIMGASRRIERIYFEISETNRAQWEMPQVKESKRQFIFDVVNEGGEAEKMELFVSFCEDTIFEMQIAAQISEPEGEPETDEDEGAGAAEAGAEGAEEGAAGLEGTAATAAAGATARVVAAAGRALRGLSYRSLRRRVRRLRRLTAREAATAVAALLWAAVTRAGAAGAGAAAGALGLLWGSLFGGGLVEGAKKVTVTELLAGMPDPTSDEVHGEQPAGPGGDADGEGASEGAGDAAEGAGDEEEAVHEAGPGGADGAVAVTDGGPFRPEGAGGLGDMGDTTPAEPPTPEGSPILKRKLGVDGVEEELPPEPEPEPEPELEPEKADAENGEKEEVPEPTPEPPKKQAPPSPPPKKEEAGGEFWGELEVQRVKFLNYLSRNFYTLRFLALFLAFAINFILLFYKVSDSPPGEDDMEGSAAGDVSGAGSGGSSGWGLGAGEEAEGDEDENMVYYFLEESTGYMEPALRCLSLLHTLVAFLCIIGYNCLKVPLVIFKREKELARKLEFDGLYITEQPEDDDVKGQWDRLVLNTPSFPSNYWDKFVKRKVLDKHGDIYGRERIAELLGMDLATLEITAHNERKPNPPPGLLTWLMSIDVKYQIWKFGVIFTDNSFLYLGWYMVMSLLGHYNNFFFAAHLLDIAMGVKTLRTILSSVTHNGKQLVMTVGLLAVVVYLYTVVAFNFFRKFYNKSEDEDEPDMKCDDMMTCYLFHMYVGVRAGGGIGDEIEDPAGDEYELYRVVFDITFFFFVIVILLAIIQGLIIDAFGELRDQQEQVKEDMETKCFICGIGSDYFDTTPHGFETHTLEEHNLANYMFFLMYLINKDETEHTGQESYVWKMYQERCWDFFPAGDCFRKQYEDQLS",
# RYR2_HUMAN
"MADGGEGEDEIQFLRTDDEVVLQCTATIHKEQQKLCLAAEGFGNRLCFLESTSNSKNVPPDLSICTFVLEQSLSVRALQEMLANTVEKSEGQVDVEKWKFMMKTAQGGGHRTLLYGHAILLRHSYSGMYLCCLSTSRSSTDKLAFDVGLQEDTTGEACWWTIHPASKQRSEGEKVRVGDDLILVSVSSERYLHLSYGNGSLHVDAAFQQTLWSVAPISSGSEAAQGYLIGGDVLRLLHGHMDECLTVPSGEHGEEQRRTVHYEGGAVSVHARSLWRLETLRVAWSGSHIRWGQPFRLRHVTTGKYLSLMEDKNLLLMDKEKADVKSTAFTFRSSKEKLDVGVRKEVDGMGTSEIKYGDSVCYIQHVDTGLWLTYQSVDVKSVRMGSIQRKAIMHHEGHMDDGISLSRSQHEESRTARVIRSTVFLFNRFIRGLDALSKKAKASTVDLPIESVSLSLQDLIGYFHPPDEHLEHEDKQNRLRALKNRQNLFQEEGMINLVLECIDRLHVYSSAAHFADVAGREAGESWKSILNSLYELLAALIRGNRKNCAQFSGSLDWLISRLERLEASSGILEVLHCVLVESPEALNIIKEGHIKSIISLLDKHGRNHKVLDVLCSLCVCHGVAVRSNQHLICDNLLPGRDLLLQTRLVNHVSSMRPNIFLGVSEGSAQYKKWYYELMVDHTEPFVTAEATHLRVGWASTEGYSPYPGGGEEWGGNGVGDDLFSYGFDGLHLWSGCIARTVSSPNQHLLRTDDVISCCLDLSAPSISFRINGQPVQGMFENFNIDGLFFPVVSFSAGIKVRFLLGGRHGEFKFLPPPGYAPCYEAVLPKEKLKVEHSREYKQERTYTRDLLGPTVSLTQAAFTPIPVDTSQIVLPPHLERIREKLAENIHELWVMNKIELGWQYGPVRDDNKRQHPCLVEFSKLPEQERNYNLQMSLETLKTLLALGCHVGISDEHAEDKVKKMKLPKNYQLTSGYKPAPMDLSFIKLTPSQEAMVDKLAENAHNVWARDRIRQGWTYGIQQDVKNRRNPRLVPYTLLDDRTKKSNKDSLREAVRTLLGYGYNLEAPDQDHAARAEVCSGTGERFRIFRAEKTYAVKAGRWYFEFETVTAGDMRVGWSRPGCQPDQELGSDERAFAFDGFKAQRWHQGNEHYGRSWQAGDVVGCMVDMNEHTMMFTLNGEILLDDSGSELAFKDFDVGDGFIPVCSLGVAQVGRMNFGKDVSTLKYFTICGLQEGYEPFAVNTNRDITMWLSKRLPQFLQVPSNHEHIEVTRIDGTIDSSPCLKVTQKSFGSQNSNTDIMFYRLSMPIECAEVFSKTVAGGLPGAGLFGPKNDLEDYDADSDFEVLMKTAHGHLVPDRVDKDKEATKPEFNNHKDYAQEKPSRLKQRFLLRRTKPDYSTSHSARLTEDVLADDRDDYDFLMQTSTYYYSVRIFPGQEPANVWVGWITSDFHQYDTGFDLDRVRTVTVTLGDEKGKVHESIKRSNCYMVCAGESMSPGQGRNNNGLEIGCVVDAASGLLTFIANGKELSTYYQVEPSTKLFPAVFAQATSPNVFQFELGRIKNVMPLSAGLFKSEHKNPVPQCPPRLHVQFLSHVLWSRMPNQFLKVDVSRISERQGWLVQCLDPLQFMSLHIPEENRSVDILELTEQEELLKFHYHTLRLYSAVCALGNHRVAHALCSHVDEPQLLYAIENKYMPGLLRAGYYDLLIDIHLSSYATARLMMNNEYIVPMTEETKSITLFPDENKKHGLPGIGLSTSLRPRMQFSSPSFVSISNECYQYSPEFPLDILKSKTIQMLTEAVKEGSLHARDPVGGTTEFLFVPLIKLFYTLLIMGIFHNEDLKHILQLIEPSVFKEAATPEEESDTLEKELSVDDAKLQGAGEEEAKGGKRPKEGLLQMKLPEPVKLQMCLLLQYLCDCQVRHRIEAIVAFSDDFVAKLQDNQRFRYNEVMQALNMSAALTARKTKEFRSPPQEQINMLLNFKDDKSECPCPEEIRDQLLDFHEDLMTHCGIELDEDGSLDGNSDLTIRGRLLSLVEKVTYLKKKQAEKPVESDSKKSSTLQQLISETMVRWAQESVIEDPELVRAMFVLLHRQYDGIGGLVRALPKTYTINGVSVEDTINLLASLGQIRSLLSVRMGKEEEKLMIRGLGDIMNNKVFYQHPNLMRALGMHETVMEVMVNVLGGGESKEITFPKMVANCCRFLCYFCRISRQNQKAMFDHLSYLLENSSVGLASPAMRGSTPLDVAAASVMDNNELALALREPDLEKVVRYLAGCGLQSCQMLVSKGYPDIGWNPVEGERYLDFLRFAVFCNGESVEENANVVVRLLIRRPECFGPALRGEGGNGLLAAMEEAIKIAEDPSRDGPSPNSGSSKTLDTEEEEDDTIHMGNAIMTFYSALIDLLGRCAPEMHLIHAGKGEAIRIRSILRSLIPLGDLVGVISIAFQMPTIAKDGNVVEPDMSAGFCPDHKAAMVLFLDRVYGIEVQDFLLHLLEVGFLPDLRAAASLDTAALSATDMALALNRYLCTAVLPLLTRCAPLFAGTEHHASLIDSLLHTVYRLSKGCSLTKAQRDSIEVCLLSICGQLRPSMMQHLLRRLVFDVPLLNEHAKMPLKLLTNHYERCWKYYCLPGGWGNFGAASEEELHLSRKLFWGIFDALSQKKYEQELFKLALPCLSAVAGALPPDYMESNYVSMMEKQSSMDSEGNFNPQPVDTSNITIPEKLEYFINKYAEHSHDKWSMDKLANGWIYGEIYSDSSKVQPLMKPYKLLSEKEKEIYRWPIKESLKTMLAWGWRIERTREGDSMALYNRTRRISQTSQVSVDAAHGYSPRAIDMSNVTLSRDLHAMAEMMAENYHNIWAKKKKMELESKGGGNHPLLVPYDTLTAKEKAKDREKAQDILKFLQINGYAVSRGFKDLELDTPSIEKRFAYSFLQQLIRYVDEAHQYILEFDGGSRGKGEHFPYEQEIKFFAKVVLPLIDQYFKNHRLYFLSAASRPLCSGGHASNKEKEMVTSLFCKLGVLVRHRISLFGNDATSIVNCLHILGQTLDARTVMKTGLESVKSALRAFLDNAAEDLEKTMENLKQGQFTHTRNQPKGVTQIINYTTVALLPMLSSLFEHIGQHQFGEDLILEDVQVSCYRILTSLYALGTSKSIYVERQRSALGECLAAFAGAFPVAFLETHLDKHNIYSIYNTKSSRERAALSLPTNVEDVCPNIPSLEKLMEEIVELAESGIRYTQMPHVMEVILPMLCSYMSRWWEHGPENNPERAEMCCTALNSEHMNTLLGNILKIIYNNLGIDEGAWMKRLAVFSQPIINKVKPQLLKTHFLPLMEKLKKKAATVVSEEDHLKAEARGDMSEAELLILDEFTTLARDLYAFYPLLIRFVDYNRAKWLKEPNPEAEELFRMVAEVFIYWSKSHNFKREEQNFVVQNEINNMSFLITDTKSKMSKAAVSDQERKKMKRKGDRYSMQTSLIVAALKRLLPIGLNICAPGDQELIALAKNRFSLKDTEDEVRDIIRSNIHLQGKLEDPAIRWQMALYKDLPNRTDDTSDPEKTVERVLDIANVLFHLEQKSKRVGRRHYCLVEHPQRSKKAVWHKLLSKQRKRAVVACFRMAPLYNLPRHRAVNLFLQGYEKSWIETEEHYFEDKLIEDLAKPGAEPPEEDEGTKRVDPLHQLILLFSRTALTEKCKLEEDFLYMAYADIMAKSCHDEEDDDGEEEVKSFEEKEMEKQKLLYQQARLHDRGAAEMVLQTISASKGETGPMVAATLKLGIAILNGGNSTVQQKMLDYLKEKKDVGFFQSLAGLMQSCSVLDLNAFERQNKAEGLGMVTEEGSGEKVLQDDEFTCDLFRFLQLLCEGHNSDFQNYLRTQTGNNTTVNIIISTVDYLLRVQESISDFYWYYSGKDVIDEQGQRNFSKAIQVAKQVFNTLTEYIQGPCTGNQQSLAHSRLWDAVVGFLHVFAHMQMKLSQDSSQIELLKELMDLQKDMVVMLLSMLEGNVVNGTIGKQMVDMLVESSNNVEMILKFFDMFLKLKDLTSSDTFKEYDPDGKGVISKRDFHKAMESHKHYTQSETEFLLSCAETDENETLDYEEFVKRFHEPAKDIGFNVAVLLTNLSEHMPNDTRLQTFLELAESVLNYFQPFLGRIEIMGSAKRIERVYFEISESSRTQWEKPQVKESKRQFIFDVVNEGGEKEKMELFVNFCEDTIFEMQLAAQISESDLNERSANKEESEKERPEEQGPRMAFFSILTVRSALFALRYNILTLMRMLSLKSLKKQMKKVKKMTVKDMVTAFFSSYWSIFMTLLHFVASVFRGFFRIICSLLLGGSLVEGAKKIKVAELLANMPDPTQDEVRGDGEEGERKPLEAALPSEDLTDLKELTEESDLLSDIFGLDLKREGGQYKLIPHNPNAGLSDLMSNPVPMPEVQEKFQEQKAKEEEKEEKEETKSEPEKAEGEDGEKEEKAKEDKGKQKLRQLHTHRYGEPEVPESAFWKKIIAYQQKLLNYFARNFYNMRMLALFVAFAINFILLFYKVSTSSVVEGKELPTRSSSENAKVTSLDSSSHRIIAVHYVLEESSGYMEPTLRILAILHTVISFFCIIGYYCLKVPLVIFKREKEVARKLEFDGLYITEQPSEDDIKGQWDRLVINTQSFPNNYWDKFVKRKVMDKYGEFYGRDRISELLGMDKAALDFSDAREKKKPKKDSSLSAVLNSIDVKYQMWKLGVVFTDNSFLYLAWYMTMSVLGHYNNFFFAAHLLDIAMGFKTLRTILSSVTHNGKQLVLTVGLLAVVVYLYTVVAFNFFRKFYNKSEDGDTPDMKCDDMLTCYMFHMYVGVRAGGGIGDEIEDPAGDEYEIYRIIFDITFFFFVIVILLAIIQGLIIDAFGELRDQQEQVKEDMETKCFICGIGNDYFDTVPHGFETHTLQEHNLANYLFFLMYLINKDETEHTGQESYVWKMYQERCWEFFPAGDCFRKQYEDQLN",
# RYR3_HUMAN
"MAEGGEGGEDEIQFLRTEDEVVLQCIATIHKEQRKFCLAAEGLGNRLCFLEPTSEAKYIPPDLCVCNFVLEQSLSVRALQEMLANTGENGGEGAAQGGGHRTLLYGHAVLLRHSFSGMYLTCLTTSRSQTDKLAFDVGLREHATGEACWWTIHPASKQRSEGEKVRIGDDLILVSVSSERYLHLSVSNGNIQVDASFMQTLWNVHPTCSGSSIEEGYLLGGHVVRLFHGHDECLTIPSTDQNDSQHRRIFYEAGGAGTRARSLWRVEPLRISWSGSNIRWGQAFRLRHLTTGHYLALTEDQGLILQDRAKSDTKSTAFSFRASKELKEKLDSSHKRDIEGMGVPEIKYGDSVCFVQHIASGLWVTYKAQDAKTSRLGPLKRKVILHQEGHMDDGLTLQRCQREESQAARIIRNTTALFSQFVSGNNRTAAPITLPIEEVLQTLQDLIAYFQPPEEEMRHEDKQNKLRSLKNRQNLFKEEGMLALVLNCIDRLNVYNSVAHFAGIAREESGMAWKEILNLLYKLLAALIRGNRNNCAQFSNNLDWLISKLDRLESSSGILEVLHCILTESPEALNLIAEGHIKSIISLLDKHGRNHKVLDILCSLCLCNGVAVRANQNLICDNLLPRRNLLLQTRLINDVTSIRPNIFLGVAEGSAQYKKWYFELIIDQVDPFLTAEPTHLRVGWASSSGYAPYPGGGEGWGGNGVGDDLYSYGFDGLHLWSGRIPRAVASINQHLLRSDDVVSCCLDLGVPSISFRINGQPVQGMFENFNTDGLFFPVMSFSAGVKVRFLMGGRHGEFKFLPPSGYAPCYEALLPKEKMRLEPVKEYKRDADGIRDLLGTTQFLSQASFIPCPVDTSQVILPPHLEKIRDRLAENIHELWGMNKIELGWTFGKIRDDNKRQHPCLVEFSKLPETEKNYNLQMSTETLKTLLALGCHIAHVNPAAEEDLKKVKLPKNYMMSNGYKPAPLDLSDVKLLPPQEILVDKLAENAHNVWAKDRIKQGWTYGIQQDLKNKRNPRLVPYALLDERTKKSNRDSLREAVRTFVGYGYNIEPSDQELADSAVEKVSIDKIRFFRVERSYAVRSGKWYFEFEVVTGGDMRVGWARPGCRPDVELGADDQAFVFEGNRGQRWHQGSGYFGRTWQPGDVVGCMINLDDASMIFTLNGELLITNKGSELAFADYEIENGFVPICCLGLSQIGRMNLGTDASTFKFYTMCGLQEGFEPFAVNMNRDVAMWFSKRLPTFVNVPKDHPHIEVMRIDGTMDSPPCLKVTHKTFGTQNSNADMIYCRLSMPVECHSSFSHSPCLDSEAFQKRKQMQEILSHTTTQCYYAIRIFAGQDPSCVWVGWVTPDYHLYSEKFDLNKNCTVTVTLGDERGRVHESVKRSNCYMVWGGDIVASSQRSNRSNVDLEIGCLVDLAMGMLSFSANGKELGTCYQVEPNTKVFPAVFLQPTSTSLFQFELGKLKNAMPLSAAIFRSEEKNPVPQCPPRLDVQTIQPVLWSRMPNSFLKVETERVSERHGWVVQCLEPLQMMALHIPEENRCVDILELCEQEDLMRFHYHTLRLYSAVCALGNSRVAYALCSHVDLSQLFYAIDNKYLPGLLRSGFYDLLISIHLASAKERKLMMKNEYIIPITSTTRNIRLFPDESKRHGLPGVGLRTCLKPGFRFSTPCFVVTGEDHQKQSPEIPLESLRTKALSMLTEAVQCSGAHIRDPVGGSVEFQFVPVLKLIGTLLVMGVFDDDDVRQILLLIDPSVFGEHSAGTEEGAEKEEVTQVEEKAVEAGEKAGKEAPVKGLLQTRLPESVKLQMCELLSYLCDCELQHRVEAIVAFGDIYVSKLQANQKFRYNELMQALNMSAALTARKTKEFRSPPQEQINMLLNFQLGENCPCPEEIREELYDFHEDLLLHCGVPLEEEEEEEEDTSWTGKLCALVYKIKGPPKPEKEQPTEEEERCPTTLKELISQTMICWAQEDQIQDSELVRMMFNLLRRQYDSIGELLQALRKTYTISHTSVSDTINLLAALGQIRSLLSVRMGKEEELLMINGLGDIMNNKVFYQHPNLMRVLGMHETVMEVMVNVLGTEKSQIAFPKMVASCCRFLCYFCRISRQNQKAMFEHLSYLLENSSVGLASPSMRGSTPLDVAASSVMDNNELALSLEEPDLEKVVTYLAGCGLQSCPMLLAKGYPDVGWNPIEGERYLSFLRFAVFVNSESVEENASVVVKLLIRRPECFGPALRGEGGNGLLAAMQGAIKISENPALDLPSQGYKREVSTGDDEEEEEIVHMGNAIMSFYSALIDLLGRCAPEMHLIQTGKGEAIRIRSILRSLVPTEDLVGIISIPLKLPSLNKDGSVSEPDMAANFCPDHKAPMVLFLDRVYGIKDQTFLLHLLEVGFLPDLRASASLDTVSLSTTEAALALNRYICSAVLPLLTRCAPLFAGTEHCTSLIDSTLQTIYRLSKGRSLTKAQRDTIEECLLAICNHLRPSMLQQLLRRLVFDVPQLNEYCKMPLKLLTNHYEQCWKYYCLPSGWGSYGLAVEEELHLTEKLFWGIFDSLSHKKYDPDLFRMALPCLSAIAGALPPDYLDTRITATLEKQISVDADGNFDPKPINTMNFSLPEKLEYIVTKYAEHSHDKWACDKSQSGWKYGISLDENVKTHPLIRPFKTLTEKEKEIYRWPARESLKTMLAVGWTVERTKEGEALVQQRENEKLRSVSQANQGNSYSPAPLDLSNVVLSRELQGMVEVVAENYHNIWAKKKKLELESKGGGSHPLLVPYDTLTAKEKFKDREKAQDLFKFLQVNGIIVSRGMKDMELDASSMEKRFAYKFLKKILKYVDSAQEFIAHLEAIVSSGKTEKSPRDQEIKFFAKVLLPLVDQYFTSHCLYFLSSPLKPLSSSGYASHKEKEMVAGLFCKLAALVRHRISLFGSDSTTMVSCLHILAQTLDTRTVMKSGSELVKAGLRAFFENAAEDLEKTSENLKLGKFTHSRTQIKGVSQNINYTTVALLPILTSIFEHVTQHQFGMDLLLGDVQISCYHILCSLYSLGTGKNIYVERQRPALGECLASLAAAIPVAFLEPTLNRYNPLSVFNTKTPRERSILGMPDTVEDMCPDIPQLEGLMKEINDLAESGARYTEMPHVIEVILPMLCNYLSYWWERGPENLPPSTGPCCTKVTSEHLSLILGNILKIINNNLGIDEASWMKRIAVYAQPIISKARPDLLRSHFIPTLEKLKKKAVKTVQEEEQLKADGKGDTQEAELLILDEFAVLCRDLYAFYPMLIRYVDNNRSNWLKSPDADSDQLFRMVAEVFILWCKSHNFKREEQNFVIQNEINNLAFLTGDSKSKMSKAMQVKSGGQDQERKKTKRRGDLYSIQTSLIVAALKKMLPIGLNMCTPGDQELISLAKSRYSHRDTDEEVREHLRNNLHLQEKSDDPAVKWQLNLYKDVLKSEEPFNPEKTVERVQRISAAVFHLEQVEQPLRSKKAVWHKLLSKQRKRAVVACFRMAPLYNLPRHRSINLFLHGYQRFWIETEEYSFEEKLVQDLAKSPKVEEEEEEETEKQPDPLHQIILYFSRNALTERSKLEDDPLYTSYSSMMAKSCQSGEDEEEDEDKEKTFEEKEMEKQKTLYQQARLHERGAAEMVLQMISASKGEMSPMVVETLKLGIAILNGGNAGVQQKMLDYLKEKKDAGFFQSLSGLMQSCSVLDLNAFERQNKAEGLGMVTEEGTLIVRERGEKVLQNDEFTRDLFRFLQLLCEGHNSDFQNFLRTQMGNTTTVNVIISTVDYLLRLQESISDFYWYYSGKDIIDESGQHNFSKALAVTKQIFNSLTEYIQGPCIGNQQSLAHSRLWDAVVGFLHVFANMQMKLSQDSSQIELLKELLDLLQDMVVMLLSLLEGNVVNGTIGKQMVDTLVESSTNVEMILKFFDMFLKLKDLTSSDTFKEYDPDGKGIISKKEFQKAMEGQKQYTQSEIDFLLSCAEADENDMFNYVDFVDRFHEPAKDIGFNVAVLLTNLSEHMPNDSRLKCLLDPAESVLNYFEPYLGRIEIMGGAKKIERVYFEISESSRTQWEKPQVKESKRQFIFDVVNEGGEQEKMELFVNFCEDTIFEMQLASQISESDSADRPEEEEEDEDSSYVLEIAGEEEEDGSLEPASAFAMACASVKRNVTDFLKRATLKNLRKQYRNVKKMTAKELVKVLFSFFWMLFVGLFQLLFTILGGIFQILWSTVFGGGLVEGAKNIRVTKILGDMPDPTQFGIHDDTMEAERAEVMEPGITTELVHFIKGEKGDTDIMSDLFGLHPKKEGSLKHGPEVGLGDLSEIIGKDEPPTLESTVQKKRKAQAAEMKAANEAEGKVESEKADMEDGEKEDKDKEEEQAEYLWTEVTKKKKRRCGQKVEKPEAFTANFFKGLEIYQTKLLHYLARNFYNLRFLALFVAFAINFILLFYKVTEEPLEEETEDVANLWNSFNDEEEEEAMVFFVLQESTGYMAPTLRALAIIHTIISLVCVVGYYCLKVPLVVFKREKEIARKLEFDGLYITEQPSEDDIKGQWDRLVINTPSFPNNYWDKFVKRKVINKYGDLYGAERIAELLGLDKNALDFSPVEETKAEAASLVSWLSSIDMKYHIWKLGVVFTDNSFLYLAWYTTMSVLGHYNNFFFAAHLLDIAMGFKTLRTILSSVTHNGKQLVLTVGLLAVVVYLYTVVAFNFFRKFYNKSEDDDEPDMKCDDMMTCYLFHMYVGVRAGGGIGDEIEDPAGDPYEMYRIVFDITFFFFVIVILLAIIQGLIIDAFGELRDQQEQVREDMETKCFICGIGNDYFDTTPHGFETHTLQEHNLANYLFFLMYLINKDETEHTGQESYVWKMYQERCWDFFPAGDCFRKQYEDQLG",
# TPC1_HUMAN
"MAVSLDDDVPLILTLDEGGSAPLAPSNGLGQEELPSKNGGSYAIHDSQAPSLSSGGESSPSSPAHNWEMNYQEAAIYLQEGENNDKFFTHPKDAKALAAYLFAHNHLFYLMELATALLLLLLSLCEAPAVPALRLGIYVHATLELFALMVVVFELCMKLRWLGLHTFIRHKRTMVKTSVLVVQFVEAIVVLVRQMSHVRVTRALRCIFLVDCRYCGGVRRNLRQIFQSLPPFMDILLLLLFFMIIFAILGFYLFSPNPSDPYFSTLENSIVSLFVLLTTANFPDVMMPSYSRNPWSCVFFIVYLSIELYFIMNLLLAVVFDTFNDIEKRKFKSLLLHKRTAIQHAYRLLISQRRPAGISYRQFEGLMRFYKPRMSARERYLTFKALNQNNTPLLSLKDFYDIYEVAALKWKAKKNREHWFDELPRTALLIFKGINILVKSKAFQYFMYLVVAVNGVWILVETFMLKGGNFFSKHVPWSYLVFLTIYGVELFLKVAGLGPVEYLSSGWNLFDFSVTVFAFLGLLALALNMEPFYFIVVLRPLQLLRLFKLKERYRNVLDTMFELLPRMASLGLTLLIFYYSFAIVGMEFFCGIVFPNCCNTSTVADAYRWRNHTVGNRTVVEEGYYYLNNFDNILNSFVTLFELTVVNNWYIIMEGVTSQTSHWSRLYFMTFYIVTMVVMTIIVAFILEAFVFRMNYSRKNQDSEVDGGITLEKEISKEELVAVLELYREARGASSDVTRLLETLSQMERYQQHSMVFLGRRSRTKSDLSLKMYQEEIQEWYEEHAREQEQQRQLSSSAAPAAQQPPGSRQRSQTVT",
# TPC2_HUMAN
"MAEPQAESEPLLGGARGGGGDWPAGLTTYRSIQVGPGAAARWDLCIDQAVVFIEDAIQYRSINHRVDASSMWLYRRYYSNVCQRTLSFTIFLILFLAFIETPSSLTSTADVRYRAAPWEPPCGLTESVEVLCLLVFAADLSVKGYLFGWAHFQKNLWLLGYLVVLVVSLVDWTVSLSLVCHEPLRIRRLLRPFFLLQNSSMMKKTLKCIRWSLPEMASVGLLLAIHLCLFTMFGMLLFAGGKQDDGQDRERLTYFQNLPESLTSLLVLLTTANNPDVMIPAYSKNRAYAIFFIVFTVIGSLFLMNLLTAIIYSQFRGYLMKSLQTSLFRRRLGTRAAFEVLSSMVGEGGAFPQAVGVKPQNLLQVLQKVQLDSSHKQAMMEKVRSYGSVLLSAEEFQKLFNELDRSVVKEHPPRPEYQSPFLQSAQFLFGHYYFDYLGNLIALANLVSICVFLVLDADVLPAERDDFILGILNCVFIVYYLLEMLLKVFALGLRGYLSYPSNVFDGLLTVVLLVLEISTLAVYRLPHPGWRPEMVGLLSLWDMTRMLNMLIVFRFLRIIPSMKLMAVVASTVLGLVQNMRAFGGILVVVYYVFAIIGINLFRGVIVALPGNSSLAPANGSAPCGSFEQLEYWANNFDDFAAALVTLWNLMVVNNWQVFLDAYRRYSGPWSKIYFVLWWLVSSVIWVNLFLALILENFLHKWDPRSHLQPLAGTPEATYQMTVELLFRDILEEPGEDELTERLSQHPHLWLCR",
# AT2A1_HUMAN
"MEAAHAKTTEECLAYFGVSETTGLTPDQVKRNLEKYGLNELPAEEGKTLWELVIEQFEDLLVRILLLAACISFVLAWFEEGEETITAFVEPFVILLILIANAIVGVWQERNAENAIEALKEYEPEMGKVYRADRKSVQRIKARDIVPGDIVEVAVGDKVPADIRILAIKSTTLRVDQSILTGESVSVIKHTEPVPDPRAVNQDKKNMLFSGTNIAAGKALGIVATTGVGTEIGKIRDQMAATEQDKTPLQQKLDEFGEQLSKVISLICVAVWLINIGHFNDPVHGGSWFRGAIYYFKIAVALAVAAIPEGLPAVITTCLALGTRRMAKKNAIVRSLPSVETLGCTSVICSDKTGTLTTNQMSVCKMFIIDKVDGDICLLNEFSITGSTYAPEGEVLKNDKPVRPGQYDGLVELATICALCNDSSLDFNEAKGVYEKVGEATETALTTLVEKMNVFNTDVRSLSKVERANACNSVIRQLMKKEFTLEFSRDRKSMSVYCSPAKSSRAAVGNKMFVKGAPEGVIDRCNYVRVGTTRVPLTGPVKEKIMAVIKEWGTGRDTLRCLALATRDTPPKREEMVLDDSARFLEYETDLTFVGVVGMLDPPRKEVTGSIQLCRDAGIRVIMITGDNKGTAIAICRRIGIFGENEEVADRAYTGREFDDLPLAEQREACRRACCFARVEPSHKSKIVEYLQSYDEITAMTGDGVNDAPALKKAEIGIAMGSGTAVAKTASEMVLADDNFSTIVAAVEEGRAIYNNMKQFIRYLISSNVGEVVCIFLTAALGLPEALIPVQLLWVNLVTDGLPATALGFNPPDLDIMDRPPRSPKEPLISGWLFFRYMAIGGYVGAATVGAAAWWFLYAEDGPHVNYSQLTHFMQCTEDNTHFEGIDCEVFEAPEPMTMALSVLVTIEMCNALNSLSENQSLLRMPPWVNIWLLGSICLSMSLHFLILYVDPLPMIFKLRALDLTQWLMVLKISLPVIGLDEILKFVARNYLEDPEDERRK",
# AT2A2_HUMAN
"MENAHTKTVEEVLGHFGVNESTGLSLEQVKKLKERWGSNELPAEEGKTLLELVIEQFEDLLVRILLLAACISFVLAWFEEGEETITAFVEPFVILLILVANAIVGVWQERNAENAIEALKEYEPEMGKVYRQDRKSVQRIKAKDIVPGDIVEIAVGDKVPADIRLTSIKSTTLRVDQSILTGESVSVIKHTDPVPDPRAVNQDKKNMLFSGTNIAAGKAMGVVVATGVNTEIGKIRDEMVATEQERTPLQQKLDEFGEQLSKVISLICIAVWIINIGHFNDPVHGGSWIRGAIYYFKIAVALAVAAIPEGLPAVITTCLALGTRRMAKKNAIVRSLPSVETLGCTSVICSDKTGTLTTNQMSVCRMFILDRVEGDTCSLNEFTITGSTYAPIGEVHKDDKPVNCHQYDGLVELATICALCNDSALDYNEAKGVYEKVGEATETALTCLVEKMNVFDTELKGLSKIERANACNSVIKQLMKKEFTLEFSRDRKSMSVYCTPNKPSRTSMSKMFVKGAPEGVIDRCTHIRVGSTKVPMTSGVKQKIMSVIREWGSGSDTLRCLALATHDNPLRREEMHLEDSANFIKYETNLTFVGCVGMLDPPRIEVASSVKLCRQAGIRVIMITGDNKGTAVAICRRIGIFGQDEDVTSKAFTGREFDELNPSAQRDACLNARCFARVEPSHKSKIVEFLQSFDEITAMTGDGVNDAPALKKAEIGIAMGSGTAVAKTASEMVLADDNFSTIVAAVEEGRAIYNNMKQFIRYLISSNVGEVVCIFLTAALGFPEALIPVQLLWVNLVTDGLPATALGFNPPDLDIMNKPPRNPKEPLISGWLFFRYLAIGCYVGAATVGAAAWWFIAADGGPRVSFYQLSHFLQCKEDNPDFEGVDCAIFESPYPMTMALSVLVTIEMCNALNSLSENQSLLRMPPWENIWLVGSICLSMSLHFLILYVEPLPLIFQITPLNVTQWLMVLKISLPVILMDETLKFVARNYLEPGKECVQPATKSCSFSACTDGISWPFVLLIMPLVIWVYSTDTNFSDMFWS",
# AT2A3_HUMAN
"MEAAHLLPAADVLRHFSVTAEGGLSPAQVTGARERYGPNELPSEEGKSLWELVLEQFEDLLVRILLLAALVSFVLAWFEEGEETTTAFVEPLVIMLILVANAIVGVWQERNAESAIEALKEYEPEMGKVIRSDRKGVQRIRARDIVPGDIVEVAVGDKVPADLRLIEIKSTTLRVDQSILTGESVSVTKHTEAIPDPRAVNQDKKNMLFSGTNITSGKAVGVAVATGLHTELGKIRSQMAAVEPERTPLQRKLDEFGRQLSHAISVICVAVWVINIGHFADPAHGGSWLRGAVYYFKIAVALAVAAIPEGLPAVITTCLALGTRRMARKNAIVRSLPSVETLGCTSVICSDKTGTLTTNQMSVCRMFVVAEADAGSCLLHEFTISGTTYTPEGEVRQGDQPVRCGQFDGLVELATICALCNDSALDYNEAKGVYEKVGEATETALTCLVEKMNVFDTDLQALSRVERAGACNTVIKQLMRKEFTLEFSRDRKSMSVYCTPTRPHPTGQGSKMFVKGAPESVIERCSSVRVGSRTAPLTPTSREQILAKIRDWGSGSDTLRCLALATRDAPPRKEDMELDDCSKFVQYETDLTFVGCVGMLDPPRPEVAACITRCYQAGIRVVMITGDNKGTAVAICRRLGIFGDTEDVAGKAYTGREFDDLSPEQQRQACRTARCFARVEPAHKSRIVENLQSFNEITAMTGDGVNDAPALKKAEIGIAMGSGTAVAKSAAEMVLSDDNFASIVAAVEEGRAIYSNMKQFIRYLISSNVGEVVCIFLTAILGLPEALIPVQLLWVNLVTDGLPATALGFNPPDLDIMEKLPRSPREALISGWLFFRYLAIGVYVGLATVAAATWWFVYDAEGPHINFYQLRNFLKCSEDNPLFAGIDCEVFESRFPTTMALSVLVTIEMCNALNSVSENQSLLRMPPWMNPWLLVAVAMSMALHFLILLVPPLPLIFQVTPLSGRQWVVVLQISLPVILLDEALKYLSRNHMHEEMSQK",
# AT2B1_HUMAN
"MGDMANNSVAYSGVKNSLKEANHDGDFGITLAELRALMELRSTDALRKIQESYGDVYGICTKLKTSPNEGLSGNPADLERREAVFGKNFIPPKKPKTFLQLVWEALQDVTLIILEIAAIVSLGLSFYQPPEGDNALCGEVSVGEEEGEGETGWIEGAAILLSVVCVVLVTAFNDWSKEKQFRGLQSRIEQEQKFTVIRGGQVIQIPVADITVGDIAQVKYGDLLPADGILIQGNDLKIDESSLTGESDHVKKSLDKDPLLLSGTHVMEGSGRMVVTAVGVNSQTGIIFTLLGAGGEEEEKKDEKKKEKKNKKQDGAIENRNKAKAQDGAAMEMQPLKSEEGGDGDEKDKKKANLPKKEKSVLQGKLTKLAVQIGKAGLLMSAITVIILVLYFVIDTFWVQKRPWLAECTPIYIQYFVKFFIIGVTVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTMNRMTVVQAYINEKHYKKVPEPEAIPPNILSYLVTGISVNCAYTSKILPPEKEGGLPRHVGNKTECALLGLLLDLKRDYQDVRNEIPEEALYKVYTFNSVRKSMSTVLKNSDGSYRIFSKGASEIILKKCFKILSANGEAKVFRPRDRDDIVKTVIEPMASEGLRTICLAFRDFPAGEPEPEWDNENDIVTGLTCIAVVGIEDPVRPEVPDAIKKCQRAGITVRMVTGDNINTARAIATKCGILHPGEDFLCLEGKDFNRRIRNEKGEIEQERIDKIWPKLRVLARSSPTDKHTLVKGIIDSTVSDQRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFTSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTLASLALATEPPTESLLLRKPYGRNKPLISRTMMKNILGHAFYQLVVVFTLLFAGEKFFDIDSGRNAPLHAPPSEHYTIVFNTFVLMQLFNEINARKIHGERNVFEGIFNNAIFCTIVLGTFVVQIIIVQFGGKPFSCSELSIEQWLWSIFLGMGTLLWGQLISTIPTSRLKFLKEAGHGTQKEEIPEEELAEDVEEIDHAERELRRGQILWFRGLNRIQTQIRVVNAFRSSLYEGLEKPESRSSIHNFMTHPEFRIEDSEPHIPLIDDTDAEDDAPTKRNSSPPPSPNKNNNAVDSGIHLTIEMNKSATSSSPGSPLHSLETSL",
# AT2B4_HUMAN
"MTNPSDRVLPANSMAESREGDFGCTVMELRKLMELRSRDALTQINVHYGGVQNLCSRLKTSPVEGLSGNPADLEKRRQVFGHNVIPPKKPKTFLELVWEALQDVTLIILEIAAIISLVLSFYRPAGEENELCGQVATTPEDENEAQAGWIEGAAILFSVIIVVLVTAFNDWSKEKQFRGLQCRIEQEQKFSIIRNGQLIQLPVAEIVVGDIAQVKYGDLLPADGILIQGNDLKIDESSLTGESDHVKKSLDKDPMLLSGTHVMEGSGRMVVTAVGVNSQTGIILTLLGVNEDDEGEKKKKGKKQGVPENRNKAKTQDGVALEIQPLNSQEGIDNEEKDKKAVKVPKKEKSVLQGKLTRLAVQIGKAGLLMSALTVFILILYFVIDNFVINRRPWLPECTPIYIQYFVKFFIIGITVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTMNRMTVVQAYIGGIHYRQIPSPDVFLPKVLDLIVNGISINSAYTSKILPPEKEGGLPRQVGNKTECALLGFVTDLKQDYQAVRNEVPEEKLYKVYTFNSVRKSMSTVIRNPNGGFRMYSKGASEIILRKCNRILDRKGEAVPFKNKDRDDMVRTVIEPMACDGLRTICIAYRDFDDTEPSWDNENEILTELTCIAVVGIEDPVRPEVPDAIAKCKQAGITVRMVTGDNINTARAIATKCGILTPGDDFLCLEGKEFNRLIRNEKGEVEQEKLDKIWPKLRVLARSSPTDKHTLVKGIIDSTVGEHRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFTSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTFASLALATEPPTESLLKRRPYGRNKPLISRTMMKNILGHAFYQLIVIFILVFAGEKFFDIDSGRKAPLHSPPSQHYTIVFNTFVLMQLFNEINSRKIHGEKNVFSGIYRNIIFCSVVLGTFICQIFIVEFGGKPFSCTSLSLSQWLWCLFIGIGELLWGQFISAIPTRSLKFLKEAGHGTTKEEITKDAEGLDEIDHAEMELRRGQILWFRGLNRIQTQIDVINTFQTGASFKGVLRRQNMGQHLDVKLVPSSSYIKVVKAFHSSLHESIQKPYNQKSIHSFMTHPEFAIEEELPRTPLLDEEEEENPDKASKFGTRVLLLDGEVTPYANTNNNAVDCNQVQLPQSDSSLQSLETSV",
# AT2B2_HUMAN
"MGDMTNSDFYSKNQRNESSHGGEFGCTMEELRSLMELRGTEAVVKIKETYGDTEAICRRLKTSPVEGLPGTAPDLEKRKQIFGQNFIPPKKPKTFLQLVWEALQDVTLIILEIAAIISLGLSFYHPPGEGNEGCATAQGGAEDEGEAEAGWIEGAAILLSVICVVLVTAFNDWSKEKQFRGLQSRIEQEQKFTVVRAGQVVQIPVAEIVVGDIAQVKYGDLLPADGLFIQGNDLKIDESSLTGESDQVRKSVDKDPMLLSGTHVMEGSGRMLVTAVGVNSQTGIIFTLLGAGGEEEEKKDKKGVKKGDGLQLPAADGAAASNAADSANASLVNGKMQDGNVDASQSKAKQQDGAAAMEMQPLKSAEGGDADDRKKASMHKKEKSVLQGKLTKLAVQIGKAGLVMSAITVIILVLYFTVDTFVVNKKPWLPECTPVYVQYFVKFFIIGVTVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTTNRMTVVQAYVGDVHYKEIPDPSSINTKTMELLINAIAINSAYTTKILPPEKEGALPRQVGNKTECGLLGFVLDLKQDYEPVRSQMPEEKLYKVYTFNSVRKSMSTVIKLPDESFRMYSKGASEIVLKKCCKILNGAGEPRVFRPRDRDEMVKKVIEPMACDGLRTICVAYRDFPSSPEPDWDNENDILNELTCICVVGIEDPVRPEVPEAIRKCQRAGITVRMVTGDNINTARAIAIKCGIIHPGEDFLCLEGKEFNRRIRNEKGEIEQERIDKIWPKLRVLARSSPTDKHTLVKGIIDSTHTEQRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFSSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTFASLALATEPPTETLLLRKPYGRNKPLISRTMMKNILGHAVYQLALIFTLLFVGEKMFQIDSGRNAPLHSPPSEHYTIIFNTFVMMQLFNEINARKIHGERNVFDGIFRNPIFCTIVLGTFAIQIVIVQFGGKPFSCSPLQLDQWMWCIFIGLGELVWGQVIATIPTSRLKFLKEAGRLTQKEEIPEEELNEDVEEIDHAERELRRGQILWFRGLNRIQTQIRVVKAFRSSLYEGLEKPESRTSIHNFMAHPEFRIEDSQPHIPLIDDTDLEEDAALKQNSSPPSSLNKNNSAIDSGINLTTDTSKSATSSSPGSPIHSLETSL",
# AT2B3_HUMAN
"MGDMANSSIEFHPKPQQQRDVPQAGGFGCTLAELRTLMELRGAEALQKIEEAYGDVSGLCRRLKTSPTEGLADNTNDLEKRRQIYGQNFIPPKQPKTFLQLVWEALQDVTLIILEVAAIVSLGLSFYAPPGEESEACGNVSGGAEDEGEAEAGWIEGAAILLSVICVVLVTAFNDWSKEKQFRGLQSRIEQEQKFTVIRNGQLLQVPVAALVVGDIAQVKYGDLLPADGVLIQANDLKIDESSLTGESDHVRKSADKDPMLLSGTHVMEGSGRMVVTAVGVNSQTGIIFTLLGAGGEEEEKKDKKGKQQDGAMESSQTKAKKQDGAVAMEMQPLKSAEGGEMEEREKKKANAPKKEKSVLQGKLTKLAVQIGKAGLVMSAITVIILVLYFVIETFVVEGRTWLAECTPVYVQYFVKFFIIGVTVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTTNRMTVVQSYLGDTHYKEIPAPSALTPKILDLLVHAISINSAYTTKILPPEKEGALPRQVGNKTECALLGFVLDLKRDFQPVREQIPEDKLYKVYTFNSVRKSMSTVIRMPDGGFRLFSKGASEILLKKCTNILNSNGELRGFRPRDRDDMVRKIIEPMACDGLRTICIAYRDFSAGQEPDWDNENEVVGDLTCIAVVGIEDPVRPEVPEAIRKCQRAGITVRMVTGDNINTARAIAAKCGIIQPGEDFLCLEGKEFNRRIRNEKGEIEQERLDKVWPKLRVLARSSPTDKHTLVKGIIDSTTGEQRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFTSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTFASLALATEPPTESLLLRKPYGRDKPLISRTMMKNILGHAVYQLAIIFTLLFVGELFFDIDSGRNAPLHSPPSEHYTIIFNTFVMMQLFNEINARKIHGERNVFDGIFSNPIFCTIVLGTFGIQIVIVQFGGKPFSCSPLSTEQWLWCLFVGVGELVWGQVIATIPTSQLKCLKEAGHGPGKDEMTDEELAEGEEEIDHAERELRRGQILWFRGLNRIQTQIRVVKAFRSSLYEGLEKPESKTSIHNFMATPEFLINDYTHNIPLIDDTDVDENEERLRAPPPPSPNQNNNAIDSGIYLTTHVTKSATSSVFSSSPGSPLHSVETSL",
# AT2C1_HUMAN
"MKVARFQKIPNGENETMIPVLTSKKASELPVSEVASILQADLQNGLNKCEVSHRRAFHGWNEFDISEDEPLWKKYISQFKNPLIMLLLASAVISVLMHQFDDAVSITVAILIVVTVAFVQEYRSEKSLEELSKLVPPECHCVREGKLEHTLARDLVPGDTVCLSVGDRVPADLRLFEAVDLSIDESSLTGETTPCSKVTAPQPAATNGDLASRSNIAFMGTLVRCGKAKGVVIGTGENSEFGEVFKMMQAEEAPKTPLQKSMDLLGKQLSFYSFGIIGIIMLVGWLLGKDILEMFTISVSLAVAAIPEGLPIVVTVTLALGVMRMVKKRAIVKKLPIVETLGCCNVICSDKTGTLTKNEMTVTHIFTSDGLHAEVTGVGYNQFGEVIVDGDVVHGFYNPAVSRIVEAGCVCNDAVIRNNTLMGKPTEGALIALAMKMGLDGLQQDYIRKAEYPFSSEQKWMAVKCVHRTQQDRPEICFMKGAYEQVIKYCTTYQSKGQTLTLTQQQRDVYQQEKARMGSAGLRVLALASGPELGQLTFLGLVGIIDPPRTGVKEAVTTLIASGVSIKMITGDSQETAVAIASRLGLYSKTSQSVSGEEIDAMDVQQLSQIVPKVAVFYRASPRHKMKIIKSLQKNGSVVAMTGDGVNDAVALKAADIGVAMGQTGTDVCKEAADMILVDDDFQTIMSAIEEGKGIYNNIKNFVRFQLSTSIAALTLISLATLMNFPNPLNAMQILWINIIMDGPPAQSLGVEPVDKDVIRKPPRNWKDSILTKNLILKILVSSIIIVCGTLFVFWRELRDNVITPRDTTMTFTCFVFFDMFNALSSRSQTKSVFEIGLCSNRMFCYAVLGSIMGQLLVIYFPPLQKVFQTESLSILDLLFLLGLTSSVCIVAEIIKKVERSREKIQKHVSSTSSSFLEV",
# AT2C2_HUMAN
"MVEGRVSEFLKKLGFSGGGRQYQALEKDEEEALIDEQSELKAIEKEKKVTALPPKEACKCQKEDLARAFCVDLHTGLSEFSVTQRRLAHGWNEFVADNSEPVWKKYLDQFKNPLILLLLGSALVSVLTKEYEDAVSIATAVLVVVTVAFIQEYRSEKSLEELTKLVPPECNCLREGKLQHLLARELVPGDVVSLSIGDRIPADIRLTEVTDLLVDESSFTGEAEPCSKTDSPLTGGGDLTTLSNIVFMGTLVQYGRGQGVVIGTGESSQFGEVFKMMQAEETPKTPLQKSMDRLGKQLTLFSFGIIGLIMLIGWSQGKQLLSMFTIGVSLAVAAIPEGLPIVVMVTLVLGVLRMAKKRVIVKKLPIVETLGCCSVLCSDKTGTLTANEMTVTQLVTSDGLRAEVSGVGYDGQGTVCLLPSKEVIKEFSNVSVGKLVEAGCVANNAVIRKNAVMGQPTEGALMALAMKMDLSDIKNSYIRKKEIPFSSEQKWMAVKCSLKTEDQEDIYFMKGALEEVIRYCTMYNNGGIPLPLTPQQRSFCLQEEKRMGSLGLRVLALASGPELGRLTFLGLVGIIDPPRVGVKEAVQVLSESGVSVKMITGDALETALAIGRNIGLCNGKLQAMSGEEVDSVEKGELADRVGKVSVFFRTSPKHKLKIIKALQESGAIVAMTGDGVNDAVALKSADIGIAMGQTGTDVSKEAANMILVDDDFSAIMNAVEEGKGIFYNIKNFVRFQLSTSISALSLITLSTVFNLPSPLNAMQILWINIIMDGPPAQSLGVEPVDKDAFRQPPRSVRDTILSRALILKILMSAAIIISGTLFIFWKEMPEDRASTPRTTTMTFTCFVFFDLFNALTCRSQTKLIFEIGFLRNHMFLYSVLGSILGQLAVIYIPPLQRVFQTENLGALDLLFLTGLASSVFILSELLKLCEKYCCSPKRVQMHPEDV",
# NAC1_HUMAN
"MYNMRRLSLSPTFSMGFHLLVTVSLLFSHVDHVIAETEMEGEGNETGECTGSYYCKKGVILPIWEPQDPSFGDKIARATVYFVAMVYMFLGVSIIADRFMSSIEVITSQEKEITIKKPNGETTKTTVRIWNETVSNLTLMALGSSAPEILLSVIEVCGHNFTAGDLGPSTIVGSAAFNMFIIIALCVYVVPDGETRKIKHLRVFFVTAAWSIFAYTWLYIILSVISPGVVEVWEGLLTFFFFPICVVFAWVADRRLLFYKYVYKRYRAGKQRGMIIEHEGDRPSSKTEIEMDGKVVNSHVENFLDGALVLEVDERDQDDEEARREMARILKELKQKHPDKEIEQLIELANYQVLSQQQKSRAFYRIQATRLMTGAGNILKRHAADQARKAVSMHEVNTEVTENDPVSKIFFEQGTYQCLENCGTVALTIIRRGGDLTNTVFVDFRTEDGTANAGSDYEFTEGTVVFKPGDTQKEIRVGIIDDDIFEEDENFLVHLSNVKVSSEASEDGILEANHVSTLACLGSPSTATVTIFDDDHAGIFTFEEPVTHVSESIGIMEVKVLRTSGARGNVIVPYKTIEGTARGGGEDFEDTCGELEFQNDEIVKTISVKVIDDEEYEKNKTFFLEIGEPRLVEMSEKKALLLNELGGFTITGKYLFGQPVFRKVHAREHPILSTVITIADEYDDKQPLTSKEEEERRIAEMGRPILGEHTKLEVIIEESYEFKSTVDKLIKKTNLALVVGTNSWREQFIEAITVSAGEDDDDDECGEEKLPSCFDYVMHFLTVFWKVLFAFVPPTEYWNGWACFIVSILMIGLLTAFIGDLASHFGCTIGLKDSVTAVVFVALGTSVPDTFASKVAATQDQYADASIGNVTGSNAVNVFLGIGVAWSIAAIYHAANGEQFKVSPGTLAFSVTLFTIFAFINVGVLLYRRRPEIGGELGGPRTAKLLTSCLFVLLWLLYIFFSSLEAYCHIKGF",
# NAC2_HUMAN
"MAPLALVGVTLLLAAPPCSGAATPTPSLPPPPANDSDTSTGGCQGSYRCQPGVLLPVWEPDDPSLGDKAARAVVYFVAMVYMFLGVSIIADRFMAAIEVITSKEKEITITKANGETSVGTVRIWNETVSNLTLMALGSSAPEILLSVIEVCGHNFQAGELGPGTIVGSAAFNMFVVIAVCIYVIPAGESRKIKHLRVFFVTASWSIFAYVWLYLILAVFSPGVVQVWEALLTLVFFPVCVVFAWMADKRLLFYKYVYKRYRTDPRSGIIIGAEGDPPKSIELDGTFVGAEAPGELGGLGPGPAEARELDASRREVIQILKDLKQKHPDKDLEQLVGIANYYALLHQQKSRAFYRIQATRLMTGAGNVLRRHAADASRRAAPAEGAGEDEDDGASRIFFEPSLYHCLENCGSVLLSVTCQGGEGNSTFYVDYRTEDGSAKAGSDYEYSEGTLVFKPGETQKELRIGIIDDDIFEEDEHFFVRLLNLRVGDAQGMFEPDGGGRPKGRLVAPLLATVTILDDDHAGIFSFQDRLLHVSECMGTVDVRVVRSSGARGTVRLPYRTVDGTARGGGVHYEDACGELEFGDDETMKTLQVKIVDDEEYEKKDNFFIELGQPQWLKRGISALLLNQGDGDRKLTAEEEEARRIAEMGKPVLGENCRLEVIIEESYDFKNTVDKLIKKTNLALVIGTHSWREQFLEAITVSAGDEEEEEDGSREERLPSCFDYVMHFLTVFWKVLFACVPPTEYCHGWACFGVSILVIGLLTALIGDLASHFGCTVGLKDSVNAVVFVALGTSIPDTFASKVAALQDQCADASIGNVTGSNAVNVFLGLGVAWSVAAVYWAVQGRPFEVRTGTLAFSVTLFTVFAFVGIAVLLYRRRPHIGGELGGPRGPKLATTALFLGLWLLYILFASLEAYCHIRGF",
# NAC3_HUMAN
"MAWLRLQPLTSAFLHFGLVTFVLFLNGLRAEAGGSGDVPSTGQNNESCSGSSDCKEGVILPIWYPENPSLGDKIARVIVYFVALIYMFLGVSIIADRFMASIEVITSQEREVTIKKPNGETSTTTIRVWNETVSNLTLMALGSSAPEILLSLIEVCGHGFIAGDLGPSTIVGSAAFNMFIIIGICVYVIPDGETRKIKHLRVFFITAAWSIFAYIWLYMILAVFSPGVVQVWEGLLTLFFFPVCVLLAWVADKRLLFYKYMHKKYRTDKHRGIIIETEGDHPKGIEMDGKMMNSHFLDGNLVPLEGKEVDESRREMIRILKDLKQKHPEKDLDQLVEMANYYALSHQQKSRAFYRIQATRMMTGAGNILKKHAAEQAKKASSMSEVHTDEPEDFISKVFFDPCSYQCLENCGAVLLTVVRKGGDMSKTMYVDYKTEDGSANAGADYEFTEGTVVLKPGETQKEFSVGIIDDDIFEEDEHFFVRLSNVRIEEEQPEEGMPPAIFNSLPLPRAVLASPCVATVTILDDDHAGIFTFECDTIHVSESIGVMEVKVLRTSGARGTVIVPFRTVEGTAKGGGEDFEDTYGELEFKNDETVKTIRVKIVDEEEYERQENFFIALGEPKWMERGISALLLSPDVTDRKLTMEEEEAKRIAEMGKPVLGEHPKLEVIIEESYEFKTTVDKLIKKTNLALVVGTHSWRDQFMEAITVSAAGDEDEDESGEERLPSCFDYVMHFLTVFWKVLFACVPPTEYCHGWACFAVSILIIGMLTAIIGDLASHFGCTIGLKDSVTAVVFVAFGTSVPDTFASKAAALQDVYADASIGNVTGSNAVNVFLGIGLAWSVAAIYWALQGQEFHVSAGTLAFSVTLFTIFAFVCISVLLYRRRPHLGGELGGPRGCKLATTWLFVSLWLLYILFATLEAYCYIKGF",
# NCLX_HUMAN
"MAGRRLNLRWALSVLCVLLMAETVSGTRGSSTGAHISPQFPASGVNQTPVVDCRKVCGLNVSDRCDFIRTNPDCHSDGGYLDYLEGIFCHFPPSLLPLAVTLYVSWLLYLFLILGVTAAKFFCPNLSAISTTLKLSHNVAGVTFLAFGNGAPDIFSALVAFSDPHTAGLALGALFGAGVLVTTVVAGGITILHPFMAASRPFFRDIVFYMVAVFLTFLMLFRGRVTLAWALGYLGLYVFYVVTVILCTWIYQRQRRGSLFCPMPVTPEILSDSEEDRVSSNTNSYDYGDEYRPLFFYQETTAQILVRALNPLDYMKWRRKSAYWKALKVFKLPVEFLLLLTVPVVDPDKDDQNWKRPLNCLHLVISPLVVVLTLQSGTYGVYEIGGLVPVWVVVVIAGTALASVTFFATSDSQPPRLHWLFAFLGFLTSALWINAAATEVVNILRSLGVVFRLSNTVLGLTLLAWGNSIGDAFSDFTLARQGYPRMAFSACFGGIIFNILVGVGLGCLLQISRSHTEVKLEPDGLLVWVLAGALGLSLVFSLVSVPLQCFQLSRVYGFCLLLFYLNFLVVALLTEFGVIHLKSM",
# NCKX4_HUMAN
"MALRGTLRPLKVRRRREMLPQQVGFVCAVLALVCCASGLFGSLGHKTASASKRVLPDTWRNRKLMAPVNGTQTAKNCTDPAIHEFPTDLFSNKERQHGAVLLHILGALYMFYALAIVCDDFFVPSLEKICERLHLSEDVAGATFMAAGSSTPELFASVIGVFITHGDVGVGTIVGSAVFNILCIIGVCGLFAGQVVRLTWWAVCRDSVYYTISVIVLIVFIYDEQIVWWEGLVLIILYVFYILIMKYNVKMQAFFTVKQKSIANGNPVNSELEAGNDFYDGSYDDPSVPLLGQVKEKPQYGKNPVVMVDEIMSSSPPKFTFPEAGLRIMITNKFGPRTRLRMASRIIINERQRLINSANGVSSKPLQNGRHENIENGNVPVENPEDPQQNQEQQPPPQPPPPEPEPVEADFLSPFSVPEARGDKVKWVFTWPLIFLLCVTIPNCSKPRWEKFFMVTFITATLWIAVFSYIMVWLVTIIGYTLGIPDVIMGITFLAAGTSVPDCMASLIVARQGLGDMAVSNTIGSNVFDILVGLGVPWGLQTMVVNYGSTVKINSRGLVYSVVLLLGSVALTVLGIHLNKWRLDRKLGVYVLVLYAIFLCFSIMIEFNVFTFVNLPMCREDD",
# AIF1_HUMAN
"MSQTRDLQGGKAFGLLKAQQEERLDEINKQFLDDPKYSSDEDLPSKLEGFKEKYMEFDLNGNGDIDIMSLKRMLEKLGVPKTHLELKKLIGEVSSGSGETFSYPDFLRMMLGKRSAILKMILMYEEKAREKEKPTGPPAKKAISELP",
# CALB1_HUMAN
"MAESHLQSSLITASQFFEIWLHFDADGSGYLEGKELQNLIQELQQARKKAGLELSPEMKTFVDQYGQRDDGKIGIVELAHVLPTEENFLLLFRCQQLKSCEEFMKTWRKYDTDHSGFIETEELKNFLKDLLEKANKTVDDTKLAEYTDLMLKLFDSNNDGKLELTEMARLLPVQENFLLKFQGIKMCGKEFNKAFELYDQDGNGYIDENELDALLKDLCEKNKQDLDINNITTYKKNIMALSDGGKLYRTDLALILCAGDN",
# CALB2_HUMAN
"MAGPQQQPPYLHLAELTASQFLEIWKHFDADGNGYIEGKELENFFQELEKARKGSGMMSKSDNFGEKMKEFMQKYDKNSDGKIEMAELAQILPTEENFLLCFRQHVGSSAEFMEAWRKYDTDRSGYIEANELKGFLSDLLKKANRPYDEPKLQEYTQTILRMFDLNGDGKLGLSEMSRLLPVQENFLLKFQGMKLTSEEFNAIFTFYDKDRSGYIDEHELDALLKDLYEKNKKEMNIQQLTNYRKSVMSLAEAGKLYRKDLEIVLCSEPPM",
# CALCR_HUMAN
"MRFTFTSRCLALFLLLNHPTPILPAFSNQTYPTIEPKPFLYVVGRKKMMDAQYKCYDRMQQLPAYQGEGPYCNRTWDGWLCWDDTPAGVLSYQFCPDYFPDFDPSEKVTKYCDEKGVWFKHPENNRTWSNYTMCNAFTPEKLKNAYVLYYLAIVGHSLSIFTLVISLGIFVFFRSLGCQRVTLHKNMFLTYILNSMIIIIHLVEVVPNGELVRRDPVSCKILHFFHQYMMACNYFWMLCEGIYLHTLIVVAVFTEKQRLRWYYLLGWGFPLVPTTIHAITRAVYFNDNCWLSVETHLLYIIHGPVMAALVVNFFFLLNIVRVLVTKMRETHEAESHMYLKAVKATMILVPLLGIQFVVFPWRPSNKMLGKIYDYVMHSLIHFQGFFVATIYCFCNNEVQTTVKRQWAQFKIQWNQRWGRRPSNRSARAAAAAAEAGDIPIYICHQELRNEPANNQGEESAEIIPLNIIEQESSA",
# CALD1_HUMAN
"MDDFERRRELRRQKREEMRLEAERIAYQRNDDDEEEAARERRRRARQERLRQKQEEESLGQVTDQVEVNAQNSVPDEEAKTTTTNTQVEGDDEAAFLERLARREERRQKRLQEALERQKEFDPTITDASLSLPSRRMQNDTAENETTEKEEKSESRQERYEIEETETVTKSYQKNDWRDAEENKKEDKEKEEEEEEKPKRGSIGENQVEVMVEEKTTESQEETVVMSLKNGQISSEEPKQEEEREQGSDEISHHEKMEEEDKERAEAERARLEAEERERIKAEQDKKIADERARIEAEEKAAAQERERREAEERERMREEEKRAAEERQRIKEEEKRAAEERQRIKEEEKRAAEERQRIKEEEKRAAEERQRARAEEEEKAKVEEQKRNKQLEEKKHAMQETKIKGEKVEQKIEGKWVNEKKAQEDKLQTAVLKKQGEEKGTKVQAKREKLQEDKPTFKKEEIKDEKIKKDKEPKEEVKSFMDRKKGFTEVKSQNGEFMTHKLKHTENTFSRPGGRASVDTKEAEGAPQVEAGKRLEELRRRRGETESEEFEKLKQKQQEAALELEELKKKREERRKVLEEEEQRRKQEEADRKLREEEEKRRLKEEIERRRAEAAEKRQKMPEDGLSDDKKPFKCFTPKGSSLKIEERAEFLNKSVQKSSGVKSTHQAAIVSKIDSRLEQYTSAIEGTKSAKPTKPAASDLPVPAEGVRNIKSMWEKGNVFSSPTAAGTPNKETAGLKVGVSSRINEWLTKTPDGNKSPAPKPSDLRPGDVSSKRNLWEKQSVDKVTSPTKV",
# CALM1_HUMAN
"MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK",
# CALR_HUMAN
"MLLSVPLLLGLLGLAVAEPAVYFKEQFLDGDGWTSRWIESKHKSDFGKFVLSSGKFYGDEEKDKGLQTSQDARFYALSASFEPFSNKGQTLVVQFTVKHEQNIDCGGGYVKLFPNSLDQTDMHGDSEYNIMFGPDICGPGTKKVHVIFNYKGKNVLINKDIRCKDDEFTHLYTLIVRPDNTYEVKIDNSQVESGSLEDDWDFLPPKKIKDPDASKPEDWDERAKIDDPTDSKPEDWDKPEHIPDPDAKKPEDWDEEMDGEWEPPVIQNPEYKGEWKPRQIDNPDYKGTWIHPEIDNPEYSPDPSIYAYDNFGVLGLDLWQVKSGTIFDNFLITNDEAYAEEFGNETWGVTKAAEKQMKDKQDEEQRLKEEEEDKKRKEEEEAEDKEDDEDKDEDEEDEEDKEEDEEEDVPGQAKDEL",
# CALR3_HUMAN
"MARALVQLWAICMLRVALATVYFQEEFLDGEHWRNRWLQSTNDSRFGHFRLSSGKFYGHKEKDKGLQTTQNGRFYAISARFKPFSNKGKTLVIQYTVKHEQKMDCGGGYIKVFPADIDQKNLNGKSQYYIMFGPDICGFDIKKVHVILHFKNKYHENKKLIRCKVDGFTHLYTLILRPDLSYDVKIDGQSIESGSIEYDWNLTSLKKETSPAESKDWEQTKDNKAQDWEKHFLDASTSKQSDWNGDLDGDWPAPMLQKPPYQDGLKPEGIHKDVWLHRKMKNTDYLTQYDLSEFENIGAIGLELWQVRSGTIFDNFLITDDEEYADNFGKATWGETKGPEREMDAIQAKEEMKKAREEEEEELLSGKINRHEHYFNQFHRRNEL",
# KCC1A_HUMAN
"MLGAVEGPRWKQAEDIRDIYDFRDVLGTGAFSEVILAEDKRTQKLVAIKCIAKEALEGKEGSMENEIAVLHKIKHPNIVALDDIYESGGHLYLIMQLVSGGELFDRIVEKGFYTERDASRLIFQVLDAVKYLHDLGIVHRDLKPENLLYYSLDEDSKIMISDFGLSKMEDPGSVLSTACGTPGYVAPEVLAQKPYSKAVDCWSIGVIAYILLCGYPPFYDENDAKLFEQILKAEYEFDSPYWDDISDSAKDFIRHLMEKDPEKRFTCEQALQHPWIAGDTALDKNIHQSVSEQIKKNFAKSKWKQAFNATAVVRHMRKLQLGTSQEGQGQTASHGELLTPVAGGPAAGCCCRDCCVEPGTELSPTLPHQL",
# KCC2A_HUMAN
"MATITCTRFTEEYQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGHHYLIFDLVTGGELFEDIVAREYYSEADASHCIQQILEAVLHCHQMGVVHRDLKPENLLLASKLKGAAVKLADFGLAIEVEGEQQAWFGFAGTPGYLSPEVLRKDPYGKPVDLWACGVILYILLVGYPPFWDEDQHRLYQQIKAGAYDFPSPEWDTVTPEAKDLINKMLTINPSKRITAAEALKHPWISHRSTVASCMHRQETVDCLKKFNARRKLKGAILTTMLATRNFSGGKSGGNKKSDGVKESSESTNTTIEDEDTKVRKQEIIKVTEQLIEAISNGDFESYTKMCDPGMTAFEPEALGNLVEGLDFHRFYFENLWSRNSKPVHTTILNPHIHLMGDESACIAYIRITQYLDAGGIPRTAQSEETRVWHRRDGKWQIVHFHRSGAPSVLPH",
# KCC2B_HUMAN
"MATTVTCTRFTDEYQLYEDIGKGAFSVVRRCVKLCTGHEYAAKIINTKKLSARDHQKLEREARICRLLKHSNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIQQILEAVLHCHQMGVVHRDLKPENLLLASKCKGAAVKLADFGLAIEVQGDQQAWFGFAGTPGYLSPEVLRKEAYGKPVDIWACGVILYILLVGYPPFWDEDQHKLYQQIKAGAYDFPSPEWDTVTPEAKNLINQMLTINPAKRITAHEALKHPWVCQRSTVASMMHRQETVECLKKFNARRKLKGAILTTMLATRNFSVGRQTTAPATMSTAASGTTMGLVEQAKSLLNKKADGVKPQTNSTKNSAAATSPKGTLPPAALEPQTTVIHNPVDGIKESSDSANTTIEDEDAKAPRVPDILSSVRRGSGAPEAEGPLPCPSPAPFSPLPAPSPRISDILNSVRRGSGTPEAEGPLSAGPPPCLSPALLGPLSSPSPRISDILNSVRRGSGTPEAEGPSPVGPPPCPSPTIPGPLPTPSRKQEIIKTTEQLIEAVNNGDFEAYAKICDPGLTSFEPEALGNLVEGMDFHRFYFENLLAKNSKPIHTTILNPHVHVIGEDAACIAYIRLTQYIDGQGRPRTSQSEETRVWHRRDGKWQNVHFHCSGAPVAPLQ",
# KCC2G_HUMAN
"MATTATCTRFTDDYQLFEELGKGAFSVVRRCVKKTSTQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIHQILESVNHIHQHDIVHRDLKPENLLLASKCKGAAVKLADFGLAIEVQGEQQAWFGFAGTPGYLSPEVLRKDPYGKPVDIWACGVILYILLVGYPPFWDEDQHKLYQQIKAGAYDFPSPEWDTVTPEAKNLINQMLTINPAKRITADQALKHPWVCQRSTVASMMHRQETVECLRKFNARRKLKGAILTTMLVSRNFSAAKSLLNKKSDGGVKKRKSSSSVHLMPQSNNKNSLVSPAQEPAPLQTAMEPQTTVVHNATDGIKGSTESCNTTTEDEDLKGRVPEGRSSRDRTAPSAGMQPQPSLCSSAMRKQEIIKITEQLIEAINNGDFEAYTKICDPGLTSFEPEALGNLVEGMDFHKFYFENLLSKNSKPIHTTILNPHVHVIGEDAACIAYIRLTQYIDGQGRPRTSQSEETRVWHRRDGKWLNVHYHCSGAPAAPLQ",
# KKCC1_HUMAN
"MEGGPAVCCQDPRAELVERVAAIDVTHLEEADGGPEPTRNGVDPPPRARAASVIPGSTSRLLPARPSLSARKLSLQERPAGSYLEAQAGPYATGPASHISPRAWRRPTIESHHVAISDAEDCVQLNQYKLQSEIGKGAYGVVRLAYNESEDRHYAMKVLSKKKLLKQYGFPRRPPPRGSQAAQGGPAKQLLPLERVYQEIAILKKLDHVNVVKLIEVLDDPAEDNLYLVFDLLRKGPVMEVPCDKPFSEEQARLYLRDVILGLEYLHCQKIVHRDIKPSNLLLGDDGHVKIADFGVSNQFEGNDAQLSSTAGTPAFMAPEAISDSGQSFSGKALDVWATGVTLYCFVYGKCPFIDDFILALHRKIKNEPVVFPEEPEISEELKDLILKMLDKNPETRIGVPDIKLHPWVTKNGEEPLPSEEEHCSVVEVTEEEVKNSVRLIPSWTTVILVKSMLRKRSFGNPFEPQARREERSMSAPGNLLVKEGFGEGGKSPELPGVQEDEAAS",
# KKCC2_HUMAN
"MSSCVSSQPSSNRAAPQDELGGRGSSSSESQKPCEALRGLSSLSIHLGMESFIVVTECEPGCAVDLGLARDRPLEADGQEVPLDTSGSQARPHLSGRKLSLQERSQGGLAAGGSLDMNGRCICPSLPYSPVSSPQSSPRLPRRPTVESHHVSITGMQDCVQLNQYTLKDEIGKGSYGVVKLAYNENDNTYYAMKVLSKKKLIRQAGFPRRPPPRGTRPAPGGCIQPRGPIEQVYQEIAILKKLDHPNVVKLVEVLDDPNEDHLYMVFELVNQGPVMEVPTLKPLSEDQARFYFQDLIKGIEYLHYQKIIHRDIKPSNLLVGEDGHIKIADFGVSNEFKGSDALLSNTVGTPAFMAPESLSETRKIFSGKALDVWAMGVTLYCFVFGQCPFMDERIMCLHSKIKSQALEFPDQPDIAEDLKDLITRMLDKNPESRIVVPEIKLHPWVTRHGAEPLPSEDENCTLVEVTEEEVENSVKHIPSLATVILVKTMIRKRSFGNPFEGSRREERSLSAPGNLLTKKPTRECESLSELKEARQRRQPPGHRPAPRGGGGSALVRGSPCVESCWAPAPGSPARMHPLRPEEAMEPE",
# CIB1_HUMAN
"MGGSGSRLSKELLAEYQDLTFLTKQEILLAHRRFCELLPQEQRSVESSLRAQVPFEQILSLPELKANPFKERICRVFSTSPAKDSLSFEDFLDLLSVFSDTATPDIKSHYAFRIFDFDDDGTLNREDLSRLVNCLTGEGEDTRLSASEMKQLIDNILEESDIDRDGTINLSEFQHVISRSPDFASSFKIVL",
# CSTN1_HUMAN
"MLRRPAPALAPAARLLLAGLLCGGGVWAARVNKHKPWLEPTYHGIVTENDNTVLLDPPLIALDKDAPLRFAESFEVTVTKEGEICGFKIHGQNVPFDAVVVDKSTGEGVIRSKEKLDCELQKDYSFTIQAYDCGKGPDGTNVKKSHKATVHIQVNDVNEYAPVFKEKSYKATVIEGKQYDSILRVEAVDADCSPQFSQICSYEIITPDVPFTVDKDGYIKNTEKLNYGKEHQYKLTVTAYDCGKKRATEDVLVKISIKPTCTPGWQGWNNRIEYEPGTGALAVFPNIHLETCDEPVASVQATVELETSHIGKGCDRDTYSEKSLHRLCGAAAGTAELLPSPSGSLNWTMGLPTDNGHDSDQVFEFNGTQAVRIPDGVVSVSPKEPFTISVWMRHGPFGRKKETILCSSDKTDMNRHHYSLYVHGCRLIFLFRQDPSEEKKYRPAEFHWKLNQVCDEEWHHYVLNVEFPSVTLYVDGTSHEPFSVTEDYPLHPSKIETQLVVGACWQEFSGVENDNETEPVTVASAGGDLHMTQFFRGNLAGLTLRSGKLADKKVIDCLYTCKEGLDLQVLEDSGRGVQIQAHPSQLVLTLEGEDLGELDKAMQHISYLNSRQFPTPGIRRLKITSTIKCFNEATCISVPPVDGYVMVLQPEEPKISLSGVHHFARAASEFESSEGVFLFPELRIISTITREVEPEGDGAEDPTVQESLVSEEIVHDLDTCEVTVEGEELNHEQESLEVDMARLQQKGIEVSSSELGMTFTGVDTMASYEEVLHLLRYRNWHARSLLDRKFKLICSELNGRYISNEFKVEVNVIHTANPMEHANHMAAQPQFVHPEHRSFVDLSGHNLANPHPFAVVPSTATVVIVVCVSFLVFMIILGVFRIRAAHRRTMRDQDTGKENEMDWDDSALTITVNPMETYEDQHSSEEEEEEEEEEESEDGEEEDDITSAESESSEEEEGEQGDPQNATRQQQLEWDDSTLSY",
# CSTN2_HUMAN
"MLPGRLCWVPLLLALGVGSGSGGGGDSRQRRLLAAKVNKHKPWIETSYHGVITENNDTVILDPPLVALDKDAPVPFAGEICAFKIHGQELPFEAVVLNKTSGEGRLRAKSPIDCELQKEYTFIIQAYDCGAGPHETAWKKSHKAVVHIQVKDVNEFAPTFKEPAYKAVVTEGKIYDSILQVEAIDEDCSPQYSQICNYEIVTTDVPFAIDRNGNIRNTEKLSYDKQHQYEILVTAYDCGQKPAAQDTLVQVDVKPVCKPGWQDWTKRIEYQPGSGSMPLFPSIHLETCDGAVSSLQIVTELQTNYIGKGCDRETYSEKSLQKLCGASSGIIDLLPSPSAATNWTAGLLVDSSEMIFKFDGRQGAKVPDGIVPKNLTDQFTITMWMKHGPSPGVRAEKETILCNSDKTEMNRHHYALYVHNCRLVFLLRKDFDQADTFRPAEFHWKLDQICDKEWHYYVINVEFPVVTLYMDGATYEPYLVTNDWPIHPSHIAMQLTVGACWQGGEVTKPQFAQFFHGSLASLTIRPGKMESQKVISCLQACKEGLDINSLESLGQGIKYHFNPSQSILVMEGDDIGNINRALQKVSYINSRQFPTAGVRRLKVSSKVQCFGEDVCISIPEVDAYVMVLQAIEPRITLRGTDHFWRPAAQFESARGVTLFPDIKIVSTFAKTEAPGDVKTTDPKSEVLEEMLHNLDFCDILVIGGDLDPRQECLELNHSELHQRHLDATNSTAGYSIYGVGSMSRYEQVLHHIRYRNWRPASLEARRFRIKCSELNGRYTSNEFNLEVSILHEDQVSDKEHVNHLIVQPPFLQSVHHPESRSSIQHSSVVPSIATVVIIISVCMLVFVVAMGVYRVRIAHQHFIQETEAAKESEMDWDDSALTITVNPMEKHEGPGHGEDETEGEEEEEAEEEMSSSSGSDDSEEEEEEEGMGRGRHGQNGARQAQLEWDDSTLPY",
# CSTN3_HUMAN
"MTLLLLPLLLASLLASCSCNKANKHKPWIEAEYQGIVMENDNTVLLNPPLFALDKDAPLRYAGEICGFRLHGSGVPFEAVILDKATGEGLIRAKEPVDCEAQKEHTFTIQAYDCGEGPDGANTKKSHKATVHVRVNDVNEFAPVFVERLYRAAVTEGKLYDRILRVEAIDGDCSPQYSQICYYEILTPNTPFLIDNDGNIENTEKLQYSGERLYKFTVTAYDCGKKRAADDAEVEIQVKPTCKPSWQGWNKRIEYAPGAGSLALFPGIRLETCDEPLWNIQATIELQTSHVAKGCDRDNYSERALRKLCGAATGEVDLLPMPGPNANWTAGLSVHYSQDSSLIYWFNGTQAVQVPLGGPSGLGSGPQDSLSDHFTLSFWMKHGVTPNKGKKEEETIVCNTVQNEDGFSHYSLTVHGCRIAFLYWPLLESARPVKFLWKLEQVCDDEWHHYALNLEFPTVTLYTDGISFDPALIHDNGLIHPPRREPALMIGACWTEEKNKEKEKGDNSTDTTQGDPLSIHHYFHGYLAGFSVRSGRLESREVIECLYACREGLDYRDFESLGKGMKVHVNPSQSLLTLEGDDVETFNHALQHVAYMNTLRFATPGVRPLRLTTAVKCFSEESCVSIPEVEGYVVVLQPDAPQILLSGTAHFARPAVDFEGTNGVPLFPDLQITCSISHQVEAKKDESWQGTVTDTRMSDEIVHNLDGCEISLVGDDLDPERESLLLDTTSLQQRGLELTNTSAYLTIAGVESITVYEEILRQARYRLRHGAALYTRKFRLSCSEMNGRYSSNEFIVEVNVLHSMNRVAHPSHVLSSQQFLHRGHQPPPEMAGHSLASSHRNSMIPSAATLIIVVCVGFLVLMVVLGLVRIHSLHRRVSGAGGPPGASSDPKDPDLFWDDSALTIIVNPMESYQNRQSCVTGAVGGQQEDEDSSDSEVADSPSSDERRIIETPPHRY",
# DCLK1_HUMAN
"MSFGRDMELEHFDERDKAQRYSRGSRVNGLPSPTHSAHCSFYRTRTLQTLSSEKKAKKVRFYRNGDRYFKGIVYAISPDRFRSFEALLADLTRTLSDNVNLPQGVRTIYTIDGLKKISSLDQLVEGESYVCGSIEPFKKLEYTKNVNPNWSVNVKTTSASRAVSSLATAKGSPSEVRENKDFIRPKLVTIIRSGVKPRKAVRILLNKKTAHSFEQVLTDITDAIKLDSGVVKRLYTLDGKQVMCLQDFFGDDDIFIACGPEKFRYQDDFLLDESECRVVKSTSYTKIASSSRRSTTKSPGPSRRSKSPASTSSVNGTPGSQLSTPRSGKSPSPSPTSPGSLRKQRSSQHGGSSTSLASTKVCSSMDENDGPGEEVSEEGFQIPATITERYKVGRTIGDGNFAVVKECVERSTAREYALKIIKKSKCRGKEHMIQNEVSILRRVKHPNIVLLIEEMDVPTELYLVMELVKGGDLFDAITSTNKYTERDASGMLYNLASAIKYLHSLNIVHRDIKPENLLVYEHQDGSKSLKLGDFGLATIVDGPLYTVCGTPTYVAPEIIAETGYGLKVDIWAAGVITYILLCGFPPFRGSGDDQEVLFDQILMGQVDFPSPYWDNVSDSAKELITMMLLVDVDQRFSAVQVLEHPWVNDDGLPENEHQLSVAGKIKKHFNTGPKPNSTAAGVSVIATTALDKERQVFRRRRNQDVRSRYKAQPAPPELNSESEDYSPSSSETVRSPNSPF",
# DOC2A_HUMAN
"MRGRRGDRMTINIQEHMAINVCPGPIRPIRQISDYFPRGPGPEGGGGGGGEAPAHLVPLALAPPAALLGATTPEDGAEVDSYDSDDATALGTLEFDLLYDRASCTLHCSILRAKGLKPMDFNGLADPYVKLHLLPGACKANKLKTKTQRNTLNPVWNEDLTYSGITDDDITHKVLRIAVCDEDKLSHNEFIGEIRVPLRRLKPSQKKHFNICLERQVPLASPSSMSAALRGISCYLKELEQAEQGQGLLEERGRILLSLSYSSRRRGLLVGILRCAHLAAMDVNGYSDPYVKTYLRPDVDKKSKHKTCVKKKTLNPEFNEEFFYEIELSTLATKTLEVTVWDYDIGKSNDFIGGVSLGPGARGEARKHWSDCLQQPDAALERWHTLTSELPPAAGALSSA",
# GPC6A_HUMAN
"MAFLIILITCFVIILATSQPCQTPDDFVAATSPGHIIIGGLFAIHEKMLSSEDSPRRPQIQECVGFEISVFLQTLAMIHSIEMINNSTLLPGVKLGYEIYDTCTEVTVAMAATLRFLSKFNCSRETVEFKCDYSSYMPRVKAVIGSGYSEITMAVSRMLNLQLMPQVGYESTAEILSDKIRFPSFLRTVPSDFHQIKAMAHLIQKSGWNWIGIITTDDDYGRLALNTFIIQAEANNVCIAFKEVLPAFLSDNTIEVRINRTLKKIILEAQVNVIVVFLRQFHVFDLFNKAIEMNINKMWIASDNWSTATKITTIPNVKKIGKVVGFAFRRGNISSFHSFLQNLHLLPSDSHKLLHEYAMHLSACAYVKDTDLSQCIFNHSQRTLAYKANKAIERNFVMRNDFLWDYAEPGLIHSIQLAVFALGYAIRDLCQARDCQNPNAFQPWELLGVLKNVTFTDGWNSFHFDAHGDLNTGYDVVLWKEINGHMTVTKMAEYDLQNDVFIIPDQETKNEFRNLKQIQSKCSKECSPGQMKKTTRSQHICCYECQNCPENHYTNQTDMPHCLLCNNKTHWAPVRSTMCFEKEVEYLNWNDSLAILLLILSLLGIIFVLVVGIIFTRNLNTPVVKSSGGLRVCYVILLCHFLNFASTSFFIGEPQDFTCKTRQTMFGVSFTLCISCILTKSLKILLAFSFDPKLQKFLKCLYRPILIIFTCTGIQVVICTLWLIFAAPTVEVNVSLPRVIILECEEGSILAFGTMLGYIAILAFICFIFAFKGKYENYNEAKFITFGMLIYFIAWITFIPIYATTFGKYVPAVEIIVILISNYGILYCTFIPKCYVIICKQEINTKSAFLKMIYSYSSHSVSSIALSPASLDSMSGNVTMTNPSSSGKSATWQKSKDLQAQAFAHICRENATSVSKTLPRKRMSSI",
# HAX1_HUMAN
"MSLFDLFRGFFGFPGPRSHRDPFFGGMTRDEDDDEEEEEEGGSWGRGNPRFHSPQHPPEEFGFGFSFSPGGGIRFHDNFGFDDLVRDFNSIFSDMGAWTLPSHPPELPGPESETPGERLREGQTLRDSMLKYPDSHQPRIFGGVLESDARSESPQPAPDWGSQRPFHRFDDVWPMDPHPRTREDNDLDSQVSQEGLGPVLQPQPKSYFKSISVTKITKPDGIVEERRTVVDSEGRTETTVTRHEADSSPRGDPESPRPPALDDAFSILDLFLGRWFRSR",
# NUCB2_HUMAN
"MRWRTILLQYCFLLITCLLTALEAVPIDIDKTKVQNIHPVESAKIEPPDTGLYYDEYLKQVIDVLETDKHFREKLQKADIEEIKSGRLSKELDLVSHHVRTKLDELKRQEVGRLRMLIKAKLDSLQDIGMDHQALLKQFDHLNHLNPDKFESTDLDMLIKAATSDLEHYDKTRHEEFKKYEMMKEHERREYLKTLNEEKRKEEESKFEEMKKKHENHPKVNHPGSKDQLKEVWEETDGLDPNDFDPKTFFKLHDVNSDGFLDEQELEALFTKELEKVYDPKNEEDDMVEMEEERLRMREHVMNEVDTNKDRLVTLEEFLKATEKKEFLEPDSWETLDQQQFFTEEELKEYENIIALQENELKKKADELQKQKEELQRQHDQLEAQKLEYHQVIQQMEQKKLQQGIPPSGPAGELKFEPHI",
# PCD12_HUMAN
"MMQLLQLLLGLLGPGGYLFLLGDCQEVTTLTVKYQVSEEVPSGTVIGKLSQELGREERRRQAGAAFQVLQLPQALPIQVDSEEGLLSTGRRLDREQLCRQWDPCLVSFDVLATGDLALIHVEIQVLDINDHQPRFPKGEQELEISESASLRTRIPLDRALDPDTGPNTLHTYTLSPSEHFALDVIVGPDETKHAELIVVKELDREIHSFFDLVLTAYDNGNPPKSGTSLVKVNVLDSNDNSPAFAESSLALEIQEDAAPGTLLIKLTATDPDQGPNGEVEFFLSKHMPPEVLDTFSIDAKTGQVILRRPLDYEKNPAYEVDVQARDLGPNPIPAHCKVLIKVLDVNDNIPSIHVTWASQPSLVSEALPKDSFIALVMADDLDSGHNGLVHCWLSQELGHFRLKRTNGNTYMLLTNATLDREQWPKYTLTLLAQDQGLQPLSAKKQLSIQISDINDNAPVFEKSRYEVSTRENNLPSLHLITIKAHDADLGINGKVSYRIQDSPVAHLVAIDSNTGEVTAQRSLNYEEMAGFEFQVIAEDSGQPMLASSVSVWVSLLDANDNAPEVVQPVLSDGKASLSVLVNASTGHLLVPIETPNGLGPAGTDTPPLATHSSRPFLLTTIVARDADSGANGEPLYSIRSGNEAHLFILNPHTGQLFVNVTNASSLIGSEWELEIVVEDQGSPPLQTRALLRVMFVTSVDHLRDSARKPGALSMSMLTVICLAVLLGIFGLILALFMSICRTEKKDNRAYNCREAESTYRQQPKRPQKHIQKADIHLVPVLRGQAGEPCEVGQSHKDVDKEAMMEAGWDPCLQAPFHLTPTLYRTLRNQGNQGAPAESREVLQDTVNLLFNHPRQRNASRENLNLPEPQPATGQPRSRPLKVAGSPTGRLAGDQGSEEAPQRPPASSATLRRQRHLNGKVSPEKESGPRQILRSLVRLSVAAFAERNPVEELTVDSPPVQQISQLLSLLHQGQFQPKPNHRGNKYLAKPGGSRSAIPDTDGPSARAGGQTDPEQEEGPLDPEEDLSVKQLLEEELSSLLDPSTGLALDRLSAPDPAWMARLSLPLTTNYRDNVISPDAAATEEPRTFQTFGKAEAPELSPTGTRLASTFVSEMSSLLEMLLEQRSSMPVEAASEALRRLSVCGRTLSLDLATSAASGMKVQGDPGGKTGTEGKSRGSSSSSRCL",
# PLPP6_HUMAN
"MPSPRRSMEGRPLGVSASSSSSSPGSPAHGGGGGGSRFEFQSLLSSRATAVDPTCARLRASESPVHRRGSFPLAAAGPSQSPAPPLPEEDRMDLNPSFLGIALRSLLAIDLWLSKKLGVCAGESSSWGSVRPLMKLLEISGHGIPWLLGTLYCLCRSDSWAGREVLMNLLFALLLDLLLVALIKGLVRRRRPAHNQMDMFVTLSVDKYSFPSGHATRAALMSRFILNHLVLAIPLRVLVVLWAFVLGLSRVMLGRHNVTDVAFGFFLGYMQYSIVDYCWLSPHNAPVLFLLWSQR",
# PDP1_HUMAN
"MPAPTQLFFPLIRNCELSRIYGTACYCHHKHLCCSSSYIPQSRLRYTPHPAYATFCRPKENWWQYTQGRRYASTPQKFYLTPPQVNSILKANEYSFKVPEFDGKNVSSILGFDSNQLPANAPIEDRRSAATCLQTRGMLLGVFDGHAGCACSQAVSERLFYYIAVSLLPHETLLEIENAVESGRALLPILQWHKHPNDYFSKEASKLYFNSLRTYWQELIDLNTGESTDIDVKEALINAFKRLDNDISLEAQVGDPNSFLNYLVLRVAFSGATACVAHVDGVDLHVANTGDSRAMLGVQEEDGSWSAVTLSNDHNAQNERELERLKLEHPKSEAKSVVKQDRLLGLLMPFRAFGDVKFKWSIDLQKRVIESGPDQLNDNEYTKFIPPNYHTPPYLTAEPEVTYHRLRPQDKFLVLATDGLWETMHRQDVVRIVGEYLTGMHHQQPIAVGGYKVTLGQMHGLLTERRTKMSSVFEDQNAATHLIRHAVGNNEFGTVDHERLSKMLSLPEELARMYRDDITIIVVQFNSHVVGAYQNQE",
# PP2BA_HUMAN
"MSEPKAIDPKLSTTDRVVKAVPFPPSHRLTAKEVFDNDGKPRVDILKAHLMKEGRLEESVALRIITEGASILRQEKNLLDIDAPVTVCGDIHGQFFDLMKLFEVGGSPANTRYLFLGDYVDRGYFSIECVLYLWALKILYPKTLFLLRGNHECRHLTEYFTFKQECKIKYSERVYDACMDAFDCLPLAALMNQQFLCVHGGLSPEINTLDDIRKLDRFKEPPAYGPMCDILWSDPLEDFGNEKTQEHFTHNTVRGCSYFYSYPAVCEFLQHNNLLSILRAHEAQDAGYRMYRKSQTTGFPSLITIFSAPNYLDVYNNKAAVLKYENNVMNIRQFNCSPHPYWLPNFMDVFTWSLPFVGEKVTEMLVNVLNICSDDELGSEEDGFDGATAAARKEVIRNKIRAIGKMARVFSVLREESESVLTLKGLTPTGMLPSGVLSGGKQTLQSATVEAIEADEAIKGFSPQHKITSFEEAKGLDRINERMPPRRDAMPSDANLNSINKALTSETNGTDSNGSNSSNIQ",
# CANB1_HUMAN
"MGNEASYPLEMCSHFDADEIKRLGKRFKKLDLDNSGSLSVEEFMSLPELQQNPLVQRVIDIFDTDGNGEVDFKEFIEGVSQFSVKGDKEQKLRFAFRIYDMDKDGYISNGELFQVLKMMVGNNLKDTQLQQIVDKTIINADKDGDGRISFEEFCAVVGGLDIHKKMVVDV",
# CANB2_HUMAN
"MGNEASYPAEMCSHFDNDEIKRLGRRFKKLDLDKSGSLSVEEFMSLPELRHNPLVRRVIDVFDTDGDGEVDFKEFILGTSQFSVKGDEEQKLRFAFSIYDMDKDGYISNGELFQVLKMMVGNNLTDWQLQQLVDKTIIILDKDGDGKISFEEFSAVVRDLEIHKKLVLIV",
# KPCD1_HUMAN
"MSAPPVLRPPSPLLPVAAAAAAAAAALVPGSGPGPAPFLAPVAAPVGGISFHLQIGLSREPVLLLQDSSGDYSLAHVREMACSIVDQKFPECGFYGMYDKILLFRHDPTSENILQLVKAASDIQEGDLIEVVLSASATFEDFQIRPHALFVHSYRAPAFCDHCGEMLWGLVRQGLKCEGCGLNYHKRCAFKIPNNCSGVRRRRLSNVSLTGVSTIRTSSAELSTSAPDEPLLQKSPSESFIGREKRSNSQSYIGRPIHLDKILMSKVKVPHTFVIHSYTRPTVCQYCKKLLKGLFRQGLQCKDCRFNCHKRCAPKVPNNCLGEVTINGDLLSPGAESDVVMEEGSDDNDSERNSGLMDDMEEAMVQDAEMAMAECQNDSGEMQDPDPDHEDANRTISPSTSNNIPLMRVVQSVKHTKRKSSTVMKEGWMVHYTSKDTLRKRHYWRLDSKCITLFQNDTGSRYYKEIPLSEILSLEPVKTSALIPNGANPHCFEITTANVVYYVGENVVNPSSPSPNNSVLTSGVGADVARMWEIAIQHALMPVIPKGSSVGTGTNLHRDISVSISVSNCQIQENVDISTVYQIFPDEVLGSGQFGIVYGGKHRKTGRDVAIKIIDKLRFPTKQESQLRNEVAILQNLHHPGVVNLECMFETPERVFVVMEKLHGDMLEMILSSEKGRLPEHITKFLITQILVALRHLHFKNIVHCDLKPENVLLASADPFPQVKLCDFGFARIIGEKSFRRSVVGTPAYLAPEVLRNKGYNRSLDMWSVGVIIYVSLSGTFPFNEDEDIHDQIQNAAFMYPPNPWKEISHEAIDLINNLLQVKMRKRYSVDKTLSHPWLQDYQTWLDLRELECKIGERYITHESDDLRWEKYAGEQGLQYPTHLINPSASHSDTPETEETEMKALGERVSIL",
# PRVA_HUMAN
"MSMTDLLNAEDIKKAVGAFSATDSFDHKKFFQMVGLKKKSADDVKKVFHMLDKDKSGFIEEDELGFILKGFSPDARDLSAKETKMLMAAGDKDGDGKIGVDEFSTLVAES",
# S10A1_HUMAN
"MGSELETAMETLINVFHAHSGKEGDKYKLSKKELKELLQTELSGFLDAQKDVDAVDKVMKELDENGDGEVDFQEYVVLVAALTVACNNFFWENS",
# S10AA_HUMAN
"MPSQMEHAMETMMFTFHKFAGDKGYLTKEDLRVLMEKEFPGFLENQKDPLAVDKIMKDLDQCRDGKVGFQSFFSLIAGLTIACNDYFVVHMKQKGKK",
# S10AB_HUMAN
"MAKISSPTETERCIESLIAVFQKYAGKDGYNYTLSKTEFLSFMNTELAAFTKNQKDPGVLDRMMKKLDTNSDGQLDFSEFLNLIGGLAMACHDSFLKAVPSQKRT",
# S10AC_HUMAN
"MTKLEEHLEGIVNIFHQYSVRKGHFDTLSKGELKQLLTKELANTIKNIKDKAVIDEIFQGLDANQDEQVDFQEFISLVAIALKAAHYHTHKE",
# S10AD_HUMAN
"MAAEPLTELEESIETVVTTFFTFARQEGRKDSLSVNEFKELVTQQLPHLLKDVGSLDEKMKSLDVNQDSELKFNEYWRLIGELAKEIRKKKDLKIRKK",
# S10AG_HUMAN
"MSDCYTELEKAVIVLVENFYKYVSKYSLVKNKISKSSFREMLQKELNHMLSDTGNRKAADKLIQNLDANHDGRISFDEYWTLIGGITGPIAKLIHEQEQQSSS",
# S10A2_HUMAN
"MMCSSLEQALAVLVTTFHKYSCQEGDKFKLSKGEMKELLHKELPSFVGEKVDEEGLKKLMGSLDENSDQQVDFQEYAVFLALITVMCNDFFQGCPDRP",
# S10A4_HUMAN
"MACPLEKALDVMVSTFHKYSGKEGDKFKLNKSELKELLTRELPSFLGKRTDEAAFQKLMSNLDSNRDNEVDFQEYCVFLSCIAMMCNEFFEGFPDKQPRKK",
# S10A6_HUMAN
"MACPLDQAIGLLVAIFHKYSGREGDKHTLSKKELKELIQKELTIGSKLQDAEIARLMEDLDRNKDQEVNFQEYVTFLGALALIYNEALKG",
# S10A7_HUMAN
"MSNTQAERSIIGMIDMFHKYTRRDDKIEKPSLLTMMKENFPNFLSACDKKGTNYLADVFEKKDKNEDKKIDFSEFLSLLGDIATDYHKQSHGAAPCSGGSQ",
# S10A8_HUMAN
"MLTELEKALNSIIDVYHKYSLIKGNFHAVYRDDLKKLLETECPQYIRKKGADVWFKELDINTDGAVNFQEFLILVIKMGVAAHKKSHEESHKE",
# S10A9_HUMAN
"MTCKMSQLERNIETIINTFHQYSVKLGHPDTLNQGEFKELVRKDLQNFLKKENKNEKVIEHIMEDLDTNADKQLSFEEFIMLMARLTWASHEKMHEGDEGPGHHHKPGLGEGTP",
# S100B_HUMAN
"MSELEKAMVALIDVFHQYSGREGDKHKLKKSELKELINNELSHFLEEIKEQEVVDKVMETLDNDGDGECDFQEFMAFVAMVTTACHEFFEHE",
# S100P_HUMAN
"MTELETAMGMIIDVFSRYSGSEGSTQTLTKGELKVLMEKELPGFLQSGKDKDAVDKLLKDLDANGDAQVDFSEFIVFVAAITSACHKYFEKAGLK",
# SMOC1_HUMAN
"MLPARCARLLTPHLLLVLVQLSPARGHRTTGPRFLISDRDPQCNLHCSRTQPKPICASDGRSYESMCEYQRAKCRDPTLGVVHRGRCKDAGQSKCRLERAQALEQAKKPQEAVFVPECGEDGSFTQVQCHTYTGYCWCVTPDGKPISGSSVQNKTPVCSGSVTDKPLSQGNSGRKDDGSKPTPTMETQPVFDGDEITAPTLWIKHLVIKDSKLNNTNIRNSEKVYSCDQERQSALEEAQQNPREGIVIPECAPGGLYKPVQCHQSTGYCWCVLVDTGRPLPGTSTRYVMPSCESDARAKTTEADDPFKDRELPGCPEGKKMEFITSLLDALTTDMVQAINSAAPTGGGRFSEPDPSHTLEERVVHWYFSQLDSNSSNDINKREMKPFKRYVKKKAKPKKCARRFTDYCDLNKDKVISLPELKGCLGVSKEGRLV",
# SMOC2_HUMAN
"MLLPQLCWLPLLAGLLPPVPAQKFSALTFLRVDQDKDKDCSLDCAGSPQKPLCASDGRTFLSRCEFQRAKCKDPQLEIAYRGNCKDVSRCVAERKYTQEQARKEFQQVFIPECNDDGTYSQVQCHSYTGYCWCVTPNGRPISGTAVAHKTPRCPGSVNEKLPQREGTGKTDDAAAPALETQPQGDEEDIASRYPTLWTEQVKSRQNKTNKNSVSSCDQEHQSALEEAKQPKNDNVVIPECAHGGLYKPVQCHPSTGYCWCVLVDTGRPIPGTSTRYEQPKCDNTARAHPAKARDLYKGRQLQGCPGAKKHEFLTSVLDALSTDMVHAASDPSSSSGRLSEPDPSHTLEERVVHWYFKLLDKNSSGDIGKKEIKPFKRFLRKKSKPKKCVKKFVEYCDVNNDKSISVQELMGCLGVAKEDGKADTKKRHTPRGHAESTSNRQPRKQG",
# SYT1_HUMAN
"MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEKFMNELHKIPLPPWALIAIAIVAVLLVLTCCFCICKKCLFKKKNKKKGKEKGGKNAINMKDVKDLGKTMKDQALKDDDAETGLTDGEEKEEPKEEEKLGKLQYSLDYDFQNNQLLVGIIQAAELPALDMGGTSDPYVKVFLLPDKKKKFETKVHRKTLNPVFNEQFTFKVPYSELGGKTLVMAVYDFDRFSKHDIIGEFKVPMNTVDFGHVTEEWRDLQSAEKEEQEKLGDICFSLRYVPTAGKLTVVILEAKNLKKMDVGGLSDPYVKIHLMQNGKRLKKKKTTIKKNTLNPYYNESFSFEVPFEQIQKVQVVVTVLDYDKIGKNDAIGKVFVGYNSTGAELRHWSDMLANPRRPIAQWHTLQVEEEVDAMLAVKK",
# TCTP_HUMAN
"MIIYRDLISHDEMFSDIYKIREIADGLCLEVEGKMVSRTEGNIDDSLIGGNASAEGPEGEGTESTVITGVDIVMNHHLQETSFTKEAYKKYIKDYMKSIKGKLEEQRPERVKPFMTGAAEQIKHILANFKNYQFFIGENMNPDGMVALLDYREDGVTPYMIFFKDGLEMEKC",
# WFS1_HUMAN
"MDSNTAPLGPSCPQPPPAPQPQARSRLNATASLEQERSERPRAPGPQAGPGPGVRDAAAPAEPQAQHTRSRERADGTGPTKGDMEIPFEEVLERAKAGDPKAQTEVGKHYLQLAGDTDEELNSCTAVDWLVLAAKQGRREAVKLLRRCLADRRGITSENEREVRQLSSETDLERAVRKAALVMYWKLNPKKKKQVAVAELLENVGQVNEHDGGAQPGPVPKSLQKQRRMLERLVSSESKNYIALDDFVEITKKYAKGVIPSSLFLQDDEDDDELAGKSPEDLPLRLKVVKYPLHAIMEIKEYLIDMASRAGMHWLSTIIPTHHINALIFFFIVSNLTIDFFAFFIPLVIFYLSFISMVICTLKVFQDSKAWENFRTLTDLLLRFEPNLDVEQAEVNFGWNHLEPYAHFLLSVFFVIFSFPIASKDCIPCSELAVITGFFTVTSYLSLSTHAEPYTRRALATEVTAGLLSLLPSMPLNWPYLKVLGQTFITVPVGHLVVLNVSVPCLLYVYLLYLFFRMAQLRNFKGTYCYLVPYLVCFMWCELSVVILLESTGLGLLRASIGYFLFLFALPILVAGLALVGVLQFARWFTSLELTKIAVTVAVCSVPLLLRWWTKASFSVVGMVKSLTRSSMVKLILVWLTAIVLFCWFYVYRSEGMKVYNSTLTWQQYGALCGPRAWKETNMARTQILCSHLEGHRVTWTGRFKYVRVTDIDNSAESAINMLPFFIGDWMRCLYGEAYPACSPGNTSTAEEELCRLKLLAKHPCHIKKFDRYKFEITVGMPFSSGADGSRSREEDDVTKDIVLRASSEFKSVLLSLRQGSLIEFSTILEGRLGSKWPVFELKAISCLNCMAQLSPTRRHVKIEHDWRSTVHGAVKFAFDFFFFPFLSAA"
]

# Add protein names corresponding to the sequences
protein_names = [
"CCG1_HUMAN",
"CCG2_HUMAN",
"CCG3_HUMAN",
"CCG4_HUMAN",
"CCG5_HUMAN",
"CCG8_HUMAN",
"CCG6_HUMAN",
"CCG7_HUMAN",
"CAC1A_HUMAN",
"CAC1B_HUMAN",
"CAC1C_HUMAN",
"CAC1D_HUMAN",
"CAC1E_HUMAN",
"CAC1F_HUMAN",
"CAC1G_HUMAN",
"CAC1H_HUMAN",
"CAC1I_HUMAN",
"CAC1S_HUMAN",
"CA2D1_HUMAN",
"CA2D2_HUMAN",
"CA2D3_HUMAN",
"CA2D4_HUMAN",
"CACB1_HUMAN",
"CACB2_HUMAN",
"CACB3_HUMAN",
"CACB4_HUMAN",
"CTSR1_HUMAN",
"CTSR2_HUMAN",
"CTSR3_HUMAN",
"CTSR4_HUMAN",
"ITPR1_HUMAN",
"ITPR2_HUMAN",
"ITPR3_HUMAN",
"RYR1_HUMAN",
"RYR2_HUMAN",
"RYR3_HUMAN",
"TPC1_HUMAN",
"TPC2_HUMAN",
"AT2A1_HUMAN",
"AT2A2_HUMAN",
"AT2A3_HUMAN",
"AT2B1_HUMAN",
"AT2B4_HUMAN",
"AT2B2_HUMAN",
"AT2B3_HUMAN",
"AT2C1_HUMAN",
"AT2C2_HUMAN",
"NAC1_HUMAN",
"NAC2_HUMAN",
"NAC3_HUMAN",
"NCLX_HUMAN",
"NCKX4_HUMAN",
"AIF1_HUMAN",
"CALB1_HUMAN",
"CALB2_HUMAN",
"CALCR_HUMAN",
"CALD1_HUMAN",
"CALM1_HUMAN",
"CALR_HUMAN",
"CALR3_HUMAN",
"KCC1A_HUMAN",
"KCC2A_HUMAN",
"KCC2B_HUMAN",
"KCC2G_HUMAN",
"KKCC1_HUMAN",
"KKCC2_HUMAN",
"CIB1_HUMAN",
"CSTN1_HUMAN",
"CSTN2_HUMAN",
"CSTN3_HUMAN",
"DCLK1_HUMAN",
"DOC2A_HUMAN",
"GPC6A_HUMAN",
"HAX1_HUMAN",
"NUCB2_HUMAN",
"PCD12_HUMAN",
"PLPP6_HUMAN",
"PDP1_HUMAN",
"PP2BA_HUMAN",
"CANB1_HUMAN",
"CANB2_HUMAN",
"KPCD1_HUMAN",
"PRVA_HUMAN",
"S10A1_HUMAN",
"S10AA_HUMAN",
"S10AB_HUMAN",
"S10AC_HUMAN",
"S10AD_HUMAN",
"S10AG_HUMAN",
"S10A2_HUMAN",
"S10A4_HUMAN",
"S10A6_HUMAN",
"S10A7_HUMAN",
"S10A8_HUMAN",
"S10A9_HUMAN",
"S100B_HUMAN",
"S100P_HUMAN",
"SMOC1_HUMAN",
"SMOC2_HUMAN",
"SYT1_HUMAN",
"TCTP_HUMAN",
"WFS1_HUMAN"
]

In [16]:
#Chaperone Genes
sequences = [
# AHSA1_HUMAN
"MAKWGEGDPRWIVEERADATNVNNWHWTERDASNWSTDKLKTLFLAVQVQNEEGKCEVTEVSKLDGEASINNRKGKLIFFYEWSVKLNWTGTSKSGVQYKGHVEIPNLSDENSVDEVEISVSLAKDEPDTNLVALMKEEGVKLLREAMGIYISTLKTEFTQGMILPTMNGESVDPVGQPALKTEERKAKPAPSKTQARPVGVKIPTCKITLKETFLTSPEELYRVFTTQELVQAFTHAPATLEADRGGKFHMVDGNVSGEFTDLVPEKHIVMKWRFKSWPEGHFATITLTFIDKNGETELCMEGRGIPAPEEERTRQGWQRYYFEGIKQTFGYGARLF",
# AHSA2_HUMAN
"MAKWGQGNPHWIVEEREDGTNVNNWRWTERDATSLSKGKFQELLVGIVVENDAGRGEINELKQVEGEASCSSRKGKLIFFYEWNIKLGWKGIVKESGVKHKGLIEIPNLSEENEVDDTEVSLSKKKGDGVILKDLMKTAGTAKVREALGDYLKALKTEFTTGMILPTKAMATQELTVKRKLSGNTLQVQASSPVALGVRIPTVALHMMELFDTTVEQLYSIFTVKELTNKKIIMKWRCGNWPEEHYAMVALNFVPTLGQTELQLKEFLSICKEENMKFCWQKQHFEEIKGSLQLTPLNG",
# AIP_HUMAN
"MADIIARLREDGIQKRVIQEGRGELPDFQDGTKATFHYRTLHSDDEGTVLDDSRARGKPMELIIGKKFKLPVWETIVCTMREGEIAQFLCDIKHVVLYPLVAKSLRNIAVGKDPLEGQRHCCGVAQMREHSSLGHADLDALQQNPQPLIFHMEMLKVESPGTYQQDPWAMTDEEKAKAVPLIHQEGNRLYREGHVKEAAAKYYDAIACLKNLQMKEQPGSPEWIQLDQQITPLLLNYCQCKLVVEEYYEVLDHCSSILNKYDDNVKAYFKRGKAHAAVWNAQEAQADFAKVLELDPALAPVVSRELRALEARIRQKDEEDKARFRGIFSH",
# AIPL1_HUMAN
"MDAALLLNVEGVKKTILHGGTGELPNFITGSRVIFHFRTMKCDEERTVIDDSRQVGQPMHIIIGNMFKLEVWEILLTSMRVHEVAEFWCDTIHTGVYPILSRSLRQMAQGKDPTEWHVHTCGLANMFAYHTLGYEDLDELQKEPQPLVFVIELLQVDAPSDYQRETWNLSNHEKMKAVPVLHGEGNRLFKLGRYEEASSKYQEAIICLRNLQTKEKPWEVQWLKLEKMINTLILNYCQCLLKKEEYYEVLEHTSDILRHHPGIVKAYYVRARAHAEVWNEAEAKADLQKVLELEPSMQKAVRRELRLLENRMAEKQEEERLRCRNMLSQGATQPPAEPPTEPPAQSSTEPPAEPPTAPSAELSAGPPAEPATEPPPSPGHSLQH",
# BAG1_HUMAN
"MAQRGGARRPRGDRERLGSRLRALRPGREPRQSEPPAQRGPPPSGRPPARSTASGHDRPTRGAAAGARRPRMKKKTRRRSTRSEELTRSEELTLSEEATWSEEATQSEEATQGEEMNRSQEVTRDEESTRSEEVTREEMAAAGLTVTVTHSNEKHDLHVTSQQGSSEPVVQDLAQVVEEVIGVPQSFQKLIFKGKSLKEMETPLSALGIQDGCRVMLIGKKNSPQEEVELKKLKHLEKSVEKIADQLEELNKELTGIQQGFLPKDLQAEALCKLDRRVKATIEQFMKILEEIDTLILPENFKDSRLKRKGLVKKVQAFLAECDTVEQNICQETERLQSTNFALAE",
# BAG2_HUMAN
"MAQAKINAKANEGRFCRSSSMADRSSRLLESLDQLELRVEALREAATAVEQEKEILLEMIHSIQNSQDMRQISDGEREELNLTANRLMGRTLTVEVSVETIRNPQQQESLKHATRIIDEVVNKFLDDLGNAKSHLMSLYSACSSEVPHGPVDQKFQSIVIGCALEDQKKIKRRLETLLRNIENSDKAIKLLEHSKGAGSKTLQQNAESRFN",
# BAG3_HUMAN
"MSAATHSPMMQVASGNGDRDPLPPGWEIKIDPQTGWPFFVDHNSRTTTWNDPRVPSEGPKETPSSANGPSREGSRLPPAREGHPVYPQLRPGYIPIPVLHEGAENRQVHPFHVYPQPGMQRFRTEAAAAAPQRSQSPLRGMPETTQPDKQCGQVAAAAAAQPPASHGPERSQSPAASDCSSSSSSASLPSSGRSSLGSHQLPRGYISIPVIHEQNVTRPAAQPSFHQAQKTHYPAQQGEYQTHQPVYHKIQGDDWEPRPLRAASPFRSSVQGASSREGSPARSSTPLHSPSPIRVHTVVDRPQQPMTHRETAPVSQPENKPESKPGPVGPELPPGHIPIQVIRKEVDSKPVSQKPPPPSEKVEVKVPPAPVPCPPPSPGPSAVPSSPKSVATEERAAPSTAPAEATPPKPGEAEAPPKHPGVLKVEAILEKVQGLEQAVDNFEGKKTDKKYLMIEEYLTKELLALDSVDPEGRADVRQARRDGVRKVQTILEKLEQKAIDVPGQVQVYELQPSNLEADQPLQAIMEMGAVAADKGKKNAGNAEDPHTETQQPEATAAATSNPSSMTDTPGNPAAP",
# BAG4_HUMAN
"MSALRRSGYGPSDGPSYGRYYGPGGGDVPVHPPPPLYPLRPEPPQPPISWRVRGGGPAETTWLGEGGGGDGYYPSGGAWPEPGRAGGSHQEQPPYPSYNSNYWNSTARSRAPYPSTYPVRPELQGQSLNSYTNGAYGPTYPPGPGANTASYSGAYYAPGYTQTSYSTEVPSTYRSSGNSPTPVSRWIYPQQDCQTEAPPLRGQVPGYPPSQNPGMTLPHYPYGDGNRSVPQSGPTVRPQEDAWASPGAYGMGGRYPWPSSAPSAPPGNLYMTESTSPWPSSGSPQSPPSPPVQQPKDSSYPYSQSDQSMNRHNFPCSVHQYESSGTVNNDDSDLLDSQVQYSAEPQLYGNATSDHPNNQDQSSSLPEECVPSDESTPPSIKKIIHVLEKVQYLEQEVEEFVGKKTDKAYWLLEEMLTKELLELDSVETGGQDSVRQARKEAVCKIQAILEKLEKKGL",
# BAG5_HUMAN
"MDMGNQHPSISRLQEIQKEVKSVEQQVIGFSGLSDDKNYKKLERILTKQLFEIDSVDTEGKGDIQQARKRAAQETERLLKELEQNANHPHRIEIQNIFEEAQSLVREKIVPFYNGGNCVTDEFEEGIQDIILRLTHVKTGGKISLRKARYHTLTKICAVQEIIEDCMKKQPSLPLSEDAHPSVAKINFVMCEVNKARGVLIALLMGVNNNETCRHLSCVLSGLIADLDALDVCGRTEIRNYRREVVEDINKLLKYLDLEEEADTTKAFDLRQNHSILKIEKVLKRMREIKNELLQAQNPSELYLSSKTELQGLIGQLDEVSLEKNPCIREARRRAVIEVQTLITYIDLKEALEKRKLFACEEHPSHKAVWNVLGNLSEIQGEVLSFDGNRTDKNYIRLEELLTKQLLALDAVDPQGEEKCKAARKQAVRLAQNILSYLDLKSDEWEY",
# BAG6_HUMAN
"MEPNDSTSTAVEEPDSLEVLVKTLDSQTRTFIVGAQMNVKEFKEHIAASVSIPSEKQRLIYQGRVLQDDKKLQEYNVGGKVIHLVERAPPQTHLPSGASSGTGSASATHGGGSPPGTRGPGASVHDRNANSYVMVGTFNLPSDGSAVDVHINMEQAPIQSEPRVRLVMAQHMIRDIQTLLSRMETLPYLQCRGGPQPQHSQPPPQPPAVTPEPVALSSQTSEPVESEAPPREPMEAEEVEERAPAQNPELTPGPAPAGPTPAPETNAPNHPSPAEYVEVLQELQRLESRLQPFLQRYYEVLGAAATTDYNNNHEGREEDQRLINLVGESLRLLGNTFVALSDLRCNLACTPPRHLHVVRPMSHYTTPMVLQQAAIPIQINVGTTVTMTGNGTRPPPTPNAEAPPPGPGQASSVAPSSTNVESSAEGAPPPGPAPPPATSHPRVIRISHQSVEPVVMMHMNIQDSGTQPGGVPSAPTGPLGPPGHGQTLGQQVPGFPTAPTRVVIARPTPPQARPSHPGGPPVSGTLQGAGLGTNASLAQMVSGLVGQLLMQPVLVAQGTPGMAPPPAPATASASAGTTNTATTAGPAPGGPAQPPPTPQPSMADLQFSQLLGNLLGPAGPGAGGSGVASPTITVAMPGVPAFLQGMTDFLQATQTAPPPPPPPPPPPPAPEQQTMPPPGSPSGGAGSPGGLGLESLSPEFFTSVVQGVLSSLLGSLGARAGSSESIAAFIQRLSGSSNIFEPGADGALGFFGALLSLLCQNFSMVDVVMLLHGHFQPLQRLQPQLRSFFHQHYLGGQEPTPSNIRMATHTLITGLEEYVRESFSLVQVQPGVDIIRTNLEFLQEQFNSIAAHVLHCTDSGFGARLLELCNQGLFECLALNLHCLGGQQMELAAVINGRIRRMSRGVNPSLVSWLTTMMGLRLQVVLEHMPVGPDAILRYVRRVGDPPQPLPEEPMEVQGAERASPEPQRENASPAPGTTAEEAMSRGPPPAPEGGSRDEQDGASAETEPWAAAVPPEWVPIIQQDIQSQRKVKPQPPLSDAYLSGMPAKRRKTMQGEGPQLLLSEAVSRAAKAAGARPLTSPESLSRDLEAPEVQESYRQQLRSDIQKRLQEDPNYSPQRFPNAQRAFADDP",
# BBS10_HUMAN
"MLSSMAAAGSVKAALQVAEVLEAIVSCCVGPEGRQVLCTKPTGEVLLSRNGGRLLEALHLEHPIARMIVDCVSSHLKKTGDGAKTFIIFLCHLLRGLHAITDREKDPLMCENIQTHGRHWKNCSRWKFISQALLTFQTQILDGIMDQYLSRHFLSIFSSAKERTLCRSSLELLLEAYFCGRVGRNNHKFISQLMCDYFFKCMTCKSGIGVFELVDDHFVELNVGVTGLPVSDSRIIAGLVLQKDFSVYRPADGDMRMVIVTETIQPLFSTSGSEFILNSEAQFQTSQFWIMEKTKAIMKHLHSQNVKLLISSVKQPDLVSYYAGVNGISVVECLSSEEVSLIRRIIGLSPFVPPQAFSQCEIPNTALVKFCKPLILRSKRYVHLGLISTCAFIPHSIVLCGPVHGLIEQHEDALHGALKMLRQLFKDLDLNYMTQTNDQNGTSSLFIYKNSGESYQAPDPGNGSIQRPYQDTVAENKDALEKTQTYLKVHSNLVIPDVELETYIPYSTPTLTPTDTFQTVETLTCLSLERNRLTDYYEPLLKNNSTAYSTRGNRIEISYENLQVTNITRKGSMLPVSCKLPNMGTSQSYLSSSMPAGCVLPVGGNFEILLHYYLLNYAKKCHQSEETMVSMIIANALLGIPKVLYKSKTGKYSFPHTYIRAVHALQTNQPLVSSQTGLESVMGKYQLLTSVLQCLTKILTIDMVITVKRHPQKVHNQDSEDEL",
# BBS12_HUMAN
"MVMACRVVNKRRHMGLQQLSSFAETGRTFLGPLKSSKFIIDEECHESVLISSTVRLLESLDLTSAVGQLLNEAVQAQNNTYRTGISTLLFLVGAWSSAVEECLHLGVPISIIVSVMSEGLNFCSEEVVSLHVPVHNIFDCMDSTKTFSQLETFSVSLCPFLQVPSDTDLIEELHGLKDVASQTLTISNLSGRPLKSYELFKPQTKVEADNNTSRTLKNSLLADTCCRQSILIHSRHFNRTDNTEGVSKPDGFQEHVTATHKTYRCNDLVELAVGLSHGDHSSMKLVEEAVQLQYQNACVQQGNCTKPFMFDISRIFTCCLPGLPETSSCVCPGYITVVSVSNNPVIKELQNQPVRIVLIEGDLTENYRHLGFNKSANIKTVLDSMRLQEDSSEELWANHVLQVLIQFKVNLVLVQGNVSERLIEKCINSKRLVIGSVNGSVMQAFAEAAGAVQVAYITQVNEDCVGDGVCVTFWRSSPLDVVDRNNRIAILLKTEGINLVTAVLTNPVTAQMQIKEDRFWTCAYRLYYALKEEKVFLGGGAVEFLCLSCLHILAEQSLKKENHACSGWLHNTSSWLASSLAIYRPTVLKFLANGWQKYLSTLLYNTANYSSEFEASTYIQHHLQNATDSGSPSSYILNEYSKLNSRIFNSDISNKLEQIPRVYDVVTPKIEAWRRALDLVLLVLQTDSEIITGHGHTQINSQELTGFLFL",
# BCS1_HUMAN
"MPLSDFILALKDNPYFGAGFGLVGVGTALALARKGVQLGLVAFRRHYMITLEVPARDRSYAWLLSWLTRHSTRTQHLSVETSYLQHESGRISTKFEFVPSPGNHFIWYRGKWIRVERSREMQMIDLQTGTPWESVTFTALGTDRKVFFNILEEARELALQQEEGKTVMYTAVGSEWRPFGYPRRRRPLNSVVLQQGLADRIVRDVQEFIDNPKWYTDRGIPYRRGYLLYGPPGCGKSSFITALAGELEHSICLLSLTDSSLSDDRLNHLLSVAPQQSLVLLEDVDAAFLSRDLAVENPVKYQGLGRLTFSGLLNALDGVASTEARIVFMTTNHVDRLDPALIRPGRVDLKEYVGYCSHWQLTQMFQRFYPGQAPSLAENFAEHVLRATNQISPAQVQGYFMLYKNDPVGAIHNAESLRR",
# CALR_HUMAN
"MLLSVPLLLGLLGLAVAEPAVYFKEQFLDGDGWTSRWIESKHKSDFGKFVLSSGKFYGDEEKDKGLQTSQDARFYALSASFEPFSNKGQTLVVQFTVKHEQNIDCGGGYVKLFPNSLDQTDMHGDSEYNIMFGPDICGPGTKKVHVIFNYKGKNVLINKDIRCKDDEFTHLYTLIVRPDNTYEVKIDNSQVESGSLEDDWDFLPPKKIKDPDASKPEDWDERAKIDDPTDSKPEDWDKPEHIPDPDAKKPEDWDEEMDGEWEPPVIQNPEYKGEWKPRQIDNPDYKGTWIHPEIDNPEYSPDPSIYAYDNFGVLGLDLWQVKSGTIFDNFLITNDEAYAEEFGNETWGVTKAAEKQMKDKQDEEQRLKEEEEDKKRKEEEEAEDKEDDEDKDEDEEDEEDKEEDEEEDVPGQAKDEL",
# CALX_HUMAN
"MEGKWLLCMLLVLGTAIVEAHDGHDDDVIDIEDDLDDVIEEVEDSKPDTTAPPSSPKVTYKAPVPTGEVYFADSFDRGTLSGWILSKAKKDDTDDEIAKYDGKWEVEEMKESKLPGDKGLVLMSRAKHHAISAKLNKPFLFDTKPLIVQYEVNFQNGIECGGAYVKLLSKTPELNLDQFHDKTPYTIMFGPDKCGEDYKLHFIFRHKNPKTGIYEEKHAKRPDADLKTYFTDKKTHLYTLILNPDNSFEILVDQSVVNSGNLLNDMTPPVNPSREIEDPEDRKPEDWDERPKIPDPEAVKPDDWDEDAPAKIPDEEATKPEGWLDDEPEYVPDPDAEKPEDWDEDMDGEWEAPQIANPRCESAPGCGVWQRPVIDNPNYKGKWKPPMIDNPSYQGIWKPRKIPNPDFFEDLEPFRMTPFSAIGLELWSMTSDIFFDNFIICADRRIVDDWANDGWGLKKAADGAAEPGVVGQMIEAAEERPWLWVVYILTVALPVFLVILFCCSGKKQTSGMEYKKTDAPQPDVKEEEEEKEEEKDKGDEEEEGEEKLEEKQKSDAEEDGGTVSQEEEDRKPKAEEDEILNRSPRNRKPRRE",
# TCPB_HUMAN
"MASLSLAPVNIFKAGADEERAETARLTSFIGAIAIGDLVKSTLGPKGMDKILLSSGRDASLMVTNDGATILKNIGVDNPAAKVLVDMSRVQDDEVGDGTTSVTVLAAELLREAESLIAKKIHPQTIIAGWREATKAAREALLSSAVDHGSDEVKFRQDLMNIAGTTLSSKLLTHHKDHFTKLAVEAVLRLKGSGNLEAIHIIKKLGGSLADSYLDEGFLLDKKIGVNQPKRIENAKILIANTGMDTDKIKIFGSRVRVDSTAKVAEIEHAEKEKMKEKVERILKHGINCFINRQLIYNYPEQLFGAAGVMAIEHADFAGVERLALVTGGEIASTFDHPELVKLGSCKLIEEVMIGEDKLIHFSGVALGEACTIVLRGATQQILDEAERSLHDALCVLAQTVKDSRTVYGGGCSEMLMAHAVTQLANRTPGKEAVAMESYAKALRMLPTIIADNAGYDSADLVAQLRAAHSEGNTTAGLDMREGTIGDMAILGITESFQVKRQVLLSAAEAAEVILRVDNIIKAAPRKRVPDHHPC",
# TCPG_HUMAN
"MMGHRPVLVLSQNTKRESGRKVQSGNINAAKTIADIIRTCLGPKSMMKMLLDPMGGIVMTNDGNAILREIQVQHPAAKSMIEISRTQDEEVGDGTTSVIILAGEMLSVAEHFLEQQMHPTVVISAYRKALDDMISTLKKISIPVDISDSDMMLNIINSSITTKAISRWSSLACNIALDAVKMVQFEENGRKEIDIKKYARVEKIPGGIIEDSCVLRGVMINKDVTHPRMRRYIKNPRIVLLDSSLEYKKGESQTDIEITREEDFTRILQMEEEYIQQLCEDIIQLKPDVVITEKGISDLAQHYLMRANITAIRRVRKTDNNRIARACGARIVSRPEELREDDVGTGAGLLEIKKIGDEYFTFITDCKDPKACTILLRGASKEILSEVERNLQDAMQVCRNVLLDPQLVPGGGASEMAVAHALTEKSKAMTGVEQWPYRAVAQALEVIPRTLIQNCGASTIRLLTSLRAKHTQENCETWGVNGETGTLVDMKELGIWEPLAVKLQTYKTAVETAVLLLRIDDIVSGHKKKGDDQSRQGGAPDAGQE",
# TCPD_HUMAN
"MPENVAPRSGATAGAAGGRGKGAYQDRDKPAQIRFSNISAAKAVADAIRTSLGPKGMDKMIQDGKGDVTITNDGATILKQMQVLHPAARMLVELSKAQDIEAGDGTTSVVIIAGSLLDSCTKLLQKGIHPTIISESFQKALEKGIEILTDMSRPVELSDRETLLNSATTSLNSKVVSQYSSLLSPMSVNAVMKVIDPATATSVDLRDIKIVKKLGGTIDDCELVEGLVLTQKVSNSGITRVEKAKIGLIQFCLSAPKTDMDNQIVVSDYAQMDRVLREERAYILNLVKQIKKTGCNVLLIQKSILRDALSDLALHFLNKMKIMVIKDIEREDIEFICKTIGTKPVAHIDQFTADMLGSAELAEEVNLNGSGKLLKITGCASPGKTVTIVVRGSNKLVIEEAERSIHDALCVIRCLVKKRALIAGGGAPEIELALRLTEYSRTLSGMESYCVRAFADAMEVIPSTLAENAGLNPISTVTELRNRHAQGEKTAGINVRKGGISNILEELVVQPLLVSVSALTLATETVRSILKIDDVVNTR",
# TCPE_HUMAN
"MASMGTLAFDEYGRPFLIIKDQDRKSRLMGLEALKSHIMAAKAVANTMRTSLGPNGLDKMMVDKDGDVTVTNDGATILSMMDVDHQIAKLMVELSKSQDDEIGDGTTGVVVLAGALLEEAEQLLDRGIHPIRIADGYEQAARVAIEHLDKISDSVLVDIKDTEPLIQTAKTTLGSKVVNSCHRQMAEIAVNAVLTVADMERRDVDFELIKVEGKVGGRLEDTKLIKGVIVDKDFSHPQMPKKVEDAKIAILTCPFEPPKPKTKHKLDVTSVEDYKALQKYEKEKFEEMIQQIKETGANLAICQWGFDDEANHLLLQNNLPAVRWVGGPEIELIAIATGGRIVPRFSELTAEKLGFAGLVQEISFGTTKDKMLVIEQCKNSRAVTIFIRGGNKMIIEEAKRSLHDALCVIRNLIRDNRVVYGGGAAEISCALAVSQEADKCPTLEQYAMRAFADALEVIPMALSENSGMNPIQTMTEVRARQVKEMNPALGIDCLHKGTNDMKQQHVIETLIGKKQQISLATQMVRMILKIDDIRKPGESEE",
# TCPZ_HUMAN
"MAAVKTLNPKAEVARAQAALAVNISAARGLQDVLRTNLGPKGTMKMLVSGAGDIKLTKDGNVLLHEMQIQHPTASLIAKVATAQDDITGDGTTSNVLIIGELLKQADLYISEGLHPRIITEGFEAAKEKALQFLEEVKVSREMDRETLIDVARTSLRTKVHAELADVLTEAVVDSILAIKKQDEPIDLFMIEIMEMKHKSETDTSLIRGLVLDHGARHPDMKKRVEDAYILTCNVSLEYEKTEVNSGFFYKSAEEREKLVKAERKFIEDRVKKIIELKRKVCGDSDKGFVVINQKGIDPFSLDALSKEGIVALRRAKRRNMERLTLACGGVALNSFDDLSPDCLGHAGLVYEYTLGEEKFTFIEKCNNPRSVTLLIKGPNKHTLTQIKDAVRDGLRAVKNAIDDGCVVPGAGAVEVAMAEALIKHKPSVKGRAQLGVQAFADALLIIPKVLAQNSGFDLQETLVKIQAEHSESGQLVGVDLNTGEPMVAAEVGVWDNYCVKKQLLHSCTVIATNILLVDEIMRAGMSSLKG",
# TCPW_HUMAN
"MAAIKAVNSKAEVARARAALAVNICAARGLQDVLRTNLGPKGTMKMLVSGAGDIKLTKDGNVLLDEMQIQHPTASLIAKVATAQDDVTGDGTTSNVLIIGELLKQADLYISEGLHPRIIAEGFEAAKIKALEVLEEVKVTKEMKRKILLDVARTSLQTKVHAELADVLTEVVVDSVLAVRRPGYPIDLFMVEIMEMKHKLGTDTKLIQGLVLDHGARHPDMKKRVEDAFILICNVSLEYEKTEVNSGFFYKTAEEKEKLVKAERKFIEDRVQKIIDLKDKVCAQSNKGFVVINQKGIDPFSLDSLAKHGIVALRRAKRRNMERLSLACGGMAVNSFEDLTVDCLGHAGLVYEYTLGEEKFTFIEECVNPCSVTLLVKGPNKHTLTQVKDAIRDGLRAIKNAIEDGCMVPGAGAIEVAMAEALVTYKNSIKGRARLGVQAFADALLIIPKVLAQNAGYDPQETLVKVQAEHVESKQLVGVDLNTGEPMVAADAGVWDNYCVKKQLLHSCTVIATNILLVDEIMRAGMSSLK",
# TCPH_HUMAN
"MMPTPVILLKEGTDSSQGIPQLVSNISACQVIAEAVRTTLGPRGMDKLIVDGRGKATISNDGATILKLLDVVHPAAKTLVDIAKSQDAEVGDGTTSVTLLAAEFLKQVKPYVEEGLHPQIIIRAFRTATQLAVNKIKEIAVTVKKADKVEQRKLLEKCAMTALSSKLISQQKAFFAKMVVDAVMMLDDLLQLKMIGIKKVQGGALEDSQLVAGVAFKKTFSYAGFEMQPKKYHNPKIALLNVELELKAEKDNAEIRVHTVEDYQAIVDAEWNILYDKLEKIHHSGAKVVLSKLPIGDVATQYFADRDMFCAGRVPEEDLKRTMMACGGSIQTSVNALSADVLGRCQVFEETQIGGERYNFFTGCPKAKTCTFILRGGAEQFMEETERSLHDAIMIVRRAIKNDSVVAGGGAIEMELSKYLRDYSRTIPGKQQLLIGAYAKALEIIPRQLCDNAGFDATNILNKLRARHAQGGTWYGVDINNEDIADNFEAFVWEPAMVRINALTAASEAACLIVSVDETIKNPRSTVDAPTAAGRGRGRGRPH",
# TCPQ_HUMAN
"MALHVPKAPGFAQMLKEGAKHFSGLEEAVYRNIQACKELAQTTRTAYGPNGMNKMVINHLEKLFVTNDAATILRELEVQHPAAKMIVMASHMQEQEVGDGTNFVLVFAGALLELAEELLRIGLSVSEVIEGYEIACRKAHEILPNLVCCSAKNLRDIDEVSSLLRTSIMSKQYGNEVFLAKLIAQACVSIFPDSGHFNVDNIRVCKILGSGISSSSVLHGMVFKKETEGDVTSVKDAKIAVYSCPFDGMITETKGTVLIKTAEELMNFSKGEENLMDAQVKAIADTGANVVVTGGKVADMALHYANKYNIMLVRLNSKWDLRRLCKTVGATALPRLTPPVLEEMGHCDSVYLSEVGDTQVVVFKHEKEDGAISTIVLRGSTDNLMDDIERAVDDGVNTFKVLTRDKRLVPGGGATEIELAKQITSYGETCPGLEQYAIKKFAEAFEAIPRALAENSGVKANEVISKLYAVHQEGNKNVGLDIEAEVPAVKDMLEAGILDTYLGKYWAIKLATNAAVTVLRVDQIIMAKPAGGPKPPSGKKDWDDDQND",
# TCPQM_HUMAN
"MDSTVPSALELPQRLALNPRESPRSPEEEEPHLLSSLAAVQTLASVIRPCYGPHGRQKFLVTMKGETVCTGCATAILRALELEHPAAWLLREAGQTQAENSGDGTAFVVLLTEALLEQAEQLLKAGLPRPQLREAYATATAEVLATLPSLAIQSLGPLEDPSWALHSVMNTHTLSPMDHLTKLVAHACWAIKELDGSFKPERVGVCALPGGTLEDSCLLPGLAISGKLCGQMATVLSGARVALFACPFGPAHPNAPATARLSSPADLAQFSKGSDQLLEKQVGQLAAAGINVAVVLGEVDEETLTLADKYGIVVIQARSWMEIIYLSEVLDTPLLPRLLPPQRPGKCQRVYRQELGDGLAVVFEWECTGTPALTVVLRGATTQGLRSAEQAVYHGIDAYFQLCQDPRLIPGAGATEMALAKMLSDKGSRLEGPSGPAFLAFAWALKYLPKTLAENAGLAVSDVMAEMSGVHQGGNLLMGVGTEGIINVAQEGVWDTLIVKAQGFRAVAEVVLQLVTVDEIVVAKKSPTHQEIWNPDSKKTKKHPPPVETKKILGLNN",
# CDC37_HUMAN
"MVDYSVWDHIEVSDDEDETHPNIDTASLFRWRHQARVERMEQFQKEKEELDRGCRECKRKVAECQRKLKELEVAEGGKAELERLQAEAQQLRKEERSWEQKLEEMRKKEKSMPWNVDTLSKDGFSKSMVNTKPEKTEEDSEEVREQKHKTFVEKYEKQIKHFGMLRRWDDSQKYLSDNVHLVCEETANYLVIWCIDLEVEEKCALMEQVAHQTIVMQFILELAKSLKVDPRACFRQFFTKIKTADRQYMEGFNDELEAFKERVRGRAKLRIEKAMKEYEEEERKKRLGPGGLDPVEVYESLPEELQKCFDVKDVQMLQDAISKMDPTDAKYHMQRCIDSGLWVPNSKASEAKEGEEAGPGDPLLEAVPKTGDEKDVSV",
# CD37L_HUMAN
"MEQPWPPPGPWSLPRAEGEAEEESDFDVFPSSPRCPQLPGGGAQMYSHGIELACQKQKEFVKSSVACKWNLAEAQQKLGSLALHNSESLDQEHAKAQTAVSELRQREEEWRQKEEALVQREKMCLWSTDAISKDVFNKSFINQDKRKDTEDEDKSESFMQKYEQKIRHFGMLSRWDDSQRFLSDHPYLVCEETAKYLILWCFHLEAEKKGALMEQIAHQAVVMQFIMEMAKNCNVDPRGCFRLFFQKAKAEEEGYFEAFKNELEAFKSRVRLYSQSQSFQPMTVQNHVPHSGVGSIGLLESLPQNPDYLQYSISTALCSLNSVVHKEDDEPKMMDTV",
# CHRD1_HUMAN
"MALLCYNRGCGQRFDPETNSDDACTYHPGVPVFHDALKGWSCCKRRTTDFSDFLSIVGCTKGRHNSEKPPEPVKPEVKTTEKKELCELKPKFQEHIIQAPKPVEAIKRPSPDEPMTNLELKISASLKQALDKLKLSSGNEENKKEEDNDEIKIGTSCKNGGCSKTYQGLESLEEVCVYHSGVPIFHEGMKYWSCCRRKTSDFNTFLAQEGCTKGKHMWTKKDAGKKVVPCRHDWHQTGGEVTISVYAKNSLPELSRVEANSTLLNVHIVFEGEKEFDQNVKLWGVIDVKRSYVTMTATKIEITMRKAEPMQWASLELPAAKKQEKQKDATTD",
# CLGN_HUMAN
"MHFQAFWLCLGLLFISINAEFMDDDVETEDFEENSEEIDVNESELSSEIKYKTPQPIGEVYFAETFDSGRLAGWVLSKAKKDDMDEEISIYDGRWEIEELKENQVPGDRGLVLKSRAKHHAISAVLAKPFIFADKPLIVQYEVNFQDGIDCGGAYIKLLADTDDLILENFYDKTSYIIMFGPDKCGEDYKLHFIFRHKHPKTGVFEEKHAKPPDVDLKKFFTDRKTHLYTLVMNPDDTFEVLVDQTVVNKGSLLEDVVPPIKPPKEIEDPNDKKPEEWDERAKIPDPSAVKPEDWDESEPAQIEDSSVVKPAGWLDDEPKFIPDPNAEKPDDWNEDTDGEWEAPQILNPACRIGCGEWKPPMIDNPKYKGVWRPPLVDNPNYQGIWSPRKIPNPDYFEDDHPFLLTSFSALGLELWSMTSDIYFDNFIICSEKEVADHWAADGWRWKIMIANANKPGVLKQLMAAAEGHPWLWLIYLVTAGVPIALITSFCWPRKVKKKHKDTEYKKTDICIPQTKGVLEQEEKEEKAALEKPMDLEEEKKQNDGEMLEKEEESEPEEKSEEEIEIIEGQEESNQSNKSGSEDEMKEADESTGSGDGPIKSVRKRRVRKD",
# CLPB_HUMAN
"MLGSLVLRRKALAPRLLLRLLRSPTLRGHGGASGRNVTTGSLGEPQWLRVATGGRPGTSPALFSGRGAATGGRQGGRFDTKCLAAATWGRLPGPEETLPGQDSWNGVPSRAGLGMCALAAALVVHCYSKSPSNKDAALLEAARANNMQEVSRLLSEGADVNAKHRLGWTALMVAAINRNNSVVQVLLAAGADPNLGDDFSSVYKTAKEQGIHSLEDGGQDGASRHITNQWTSALEFRRWLGLPAGVLITREDDFNNRLNNRASFKGCTALHYAVLADDYRTVKELLDGGANPLQRNEMGHTPLDYAREGEVMKLLRTSEAKYQEKQRKREAEERRRFPLEQRLKEHIIGQESAIATVGAAIRRKENGWYDEEHPLVFLFLGSSGIGKTELAKQTAKYMHKDAKKGFIRLDMSEFQERHEVAKFIGSPPGYVGHEEGGQLTKKLKQCPNAVVLFDEVDKAHPDVLTIMLQLFDEGRLTDGKGKTIDCKDAIFIMTSNVASDEIAQHALQLRQEALEMSRNRIAENLGDVQISDKITISKNFKENVIRPILKAHFRRDEFLGRINEIVYFLPFCHSELIQLVNKELNFWAKRAKQRHNITLLWDREVADVLVDGYNVHYGARSIKHEVERRVVNQLAAAYEQDLLPGGCTLRITVEDSDKQLLKSPELPSPQAEKRLPKLRLEIIDKDSKTRRLDIRAPLHPEKVCNTI",
# CLPX_HUMAN
"MPSCGACTCGAAAVRLITSSLASAQRGISGGRIHMSVLGRLGTFETQILQRAPLRSFTETPAYFASKDGISKDGSGDGNKKSASEGSSKKSGSGNSGKGGNQLRCPKCGDLCTHVETFVSSTRFVKCEKCHHFFVVLSEADSKKSIIKEPESAAEAVKLAFQQKPPPPPKKIYNYLDKYVVGQSFAKKVLSVAVYNHYKRIYNNIPANLRQQAEVEKQTSLTPRELEIRRREDEYRFTKLLQIAGISPHGNALGASMQQQVNQQIPQEKRGGEVLDSSHDDIKLEKSNILLLGPTGSGKTLLAQTLAKCLDVPFAICDCTTLTQAGYVGEDIESVIAKLLQDANYNVEKAQQGIVFLDEVDKIGSVPGIHQLRDVGGEGVQQGLLKLLEGTIVNVPEKNSRKLRGETVQVDTTNILFVASGAFNGLDRIISRRKNEKYLGFGTPSNLGKGRRAAAAADLANRSGESNTHQDIEEKDRLLRHVEARDLIEFGMIPEFVGRLPVVVPLHSLDEKTLVQILTEPRNAVIPQYQALFSMDKCELNVTEDALKAIARLALERKTGARGLRSIMEKLLLEPMFEVPNSDIVCVEVDKEVVEGKKEPGYIRAPTKESSEEEYDSGVEEEGWPRQADAANS",
# CRYAA_HUMAN
"MDVTIQHPWFKRTLGPFYPSRLFDQFFGEGLFEYDLLPFLSSTISPYYRQSLFRTVLDSGISEVRSDRDKFVIFLDVKHFSPEDLTVKVQDDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLSADGMLTFCGPKIQTGLDATHAERAIPVSREEKPTSAPSS",
# CRYAB_HUMAN
"MDIAIHHPWIRRPFFPFHSPSRLFDQFFGEHLLESDLFPTSTSLSPFYLRPPSFLRAPSWFDTGLSEMRLEKDRFSVNLDVKHFSPEELKVKVLGDVIEVHGKHEERQDEHGFISREFHRKYRIPADVDPLTITSSLSSDGVLTVNGPRKQVSGPERTIPITREEKPAVTAAPKK",
# DNJA1_HUMAN
"MVKETTYYDVLGVKPNATQEELKKAYRKLALKYHPDKNPNEGEKFKQISQAYEVLSDAKKRELYDKGGEQAIKEGGAGGGFGSPMDIFDMFFGGGGRMQRERRGKNVVHQLSVTLEDLYNGATRKLALQKNVICDKCEGRGGKKGAVECCPNCRGTGMQIRIHQIGPGMVQQIQSVCMECQGHGERISPKDRCKSCNGRKIVREKKILEVHIDKGMKDGQKITFHGEGDQEPGLEPGDIIIVLDQKDHAVFTRRGEDLFMCMDIQLVEALCGFQKPISTLDNRTIVITSHPGQIVKHGDIKCVLNEGMPIYRRPYEKGRLIIEFKVNFPENGFLSPDKLSLLEKLLPERKEVEETDEMDQVELVDFDPNQERRRHYNGEAYEDDEHHPRGGVQCQTS",
# DNJA2_HUMAN
"MANVADTKLYDILGVPPGASENELKKAYRKLAKEYHPDKNPNAGDKFKEISFAYEVLSNPEKRELYDRYGEQGLREGSGGGGGMDDIFSHIFGGGLFGFMGNQSRSRNGRRRGEDMMHPLKVSLEDLYNGKTTKLQLSKNVLCSACSGQGGKSGAVQKCSACRGRGVRIMIRQLAPGMVQQMQSVCSDCNGEGEVINEKDRCKKCEGKKVIKEVKILEVHVDKGMKHGQRITFTGEADQAPGVEPGDIVLLLQEKEHEVFQRDGNDLHMTYKIGLVEALCGFQFTFKHLDGRQIVVKYPPGKVIEPGCVRVVRGEGMPQYRNPFEKGDLYIKFDVQFPENNWINPDKLSELEDLLPSRPEVPNIIGETEEVELQEFDSTRGSGGGQRREAYNDSSDEESSSHHGPGVQCAHQ",
# DNJA3_HUMAN
"MAARCSTRWLLVVVGTPRLPAISGRGARPPREGVVGAWLSRKLSVPAFASSLTSCGPRALLTLRPGVSLTGTKHNPFICTASFHTSAPLAKEDYYQILGVPRNASQKEIKKAYYQLAKKYHPDTNKDDPKAKEKFSQLAEAYEVLSDEVKRKQYDAYGSAGFDPGASGSQHSYWKGGPTVDPEELFRKIFGEFSSSSFGDFQTVFDQPQEYFMELTFNQAAKGVNKEFTVNIMDTCERCNGKGNEPGTKVQHCHYCGGSGMETINTGPFVMRSTCRRCGGRGSIIISPCVVCRGAGQAKQKKRVMIPVPAGVEDGQTVRMPVGKREIFITFRVQKSPVFRRDGADIHSDLFISIAQALLGGTARAQGLYETINVTIPPGTQTDQKIRMGGKGIPRINSYGYGDHYIHIKIRVPKRLTSRQQSLILSYAEDETDVEGTVNGVTLTSSGGSTMDSSAGSKARREAGEDEEGFLSKLKKMFTS",
# DNJA4_HUMAN
"MVKETQYYDILGVKPSASPEEIKKAYRKLALKYHPDKNPDEGEKFKLISQAYEVLSDPKKRDVYDQGGEQAIKEGGSGSPSFSSPMDIFDMFFGGGGRMARERRGKNVVHQLSVTLEDLYNGVTKKLALQKNVICEKCEGVGGKKGSVEKCPLCKGRGMQIHIQQIGPGMVQQIQTVCIECKGQGERINPKDRCESCSGAKVIREKKIIEVHVEKGMKDGQKILFHGEGDQEPELEPGDVIIVLDQKDHSVFQRRGHDLIMKMKIQLSEALCGFKKTIKTLDNRILVITSKAGEVIKHGDLRCVRDEGMPIYKAPLEKGILIIQFLVIFPEKHWLSLEKLPQLEALLPPRQKVRITDDMDQVELKEFCPNEQNWRQHREAYEEDEDGPQAGVQCQTA",
# DNJB1_HUMAN
"MGKDYYQTLGLARGASDEEIKRAYRRQALRYHPDKNKEPGAEEKFKEIAEAYDVLSDPRKREIFDRYGEEGLKGSGPSGGSGGGANGTSFSYTFHGDPHAMFAEFFGGRNPFDTFFGQRNGEEGMDIDDPFSGFPMGMGGFTNVNFGRSRSAQEPARKKQDPPVTHDLRVSLEEIYSGCTKKMKISHKRLNPDGKSIRNEDKILTIEVKKGWKEGTKITFPKEGDQTSNNIPADIVFVLKDKPHNIFKRDGSDVIYPARISLREALCGCTVNVPTLDGRTIPVVFKDVIRPGMRRKVPGEGLPLPKTPEKRGDLIIEFEVIFPERIPQTSRTVLEQVLPI",
# DJB11_HUMAN
"MAPQNLSTFCLLLLYLIGAVIAGRDFYKILGVPRSASIKDIKKAYRKLALQLHPDRNPDDPQAQEKFQDLGAAYEVLSDSEKRKQYDTYGEEGLKDGHQSSHGDIFSHFFGDFGFMFGGTPRQQDRNIPRGSDIIVDLEVTLEEVYAGNFVEVVRNKPVARQAPGKRKCNCRQEMRTTQLGPGRFQMTQEVVCDECPNVKLVNEERTLEVEIEPGVRDGMEYPFIGEGEPHVDGEPGDLRFRIKVVKHPIFERRGDDLYTNVTISLVESLVGFEMDITHLDGHKVHISRDKITRPGAKLWKKGEGLPNFDNNNIKGSLIITFDVDFPKEQLTEEAREGIKQLLKQGSVQKVYNGLQGY",
# DJB12_HUMAN
"MESNKDEAERCISIALKAIQSNQPDRALRFLEKAQRLYPTPRVRALIESLNQKPQTAGDQPPPTDTTHATHRKAGGTDAPSANGEAGGESTKGYTAEQVAAVKRVKQCKDYYEILGVSRGASDEDLKKAYRRLALKFHPDKNHAPGATEAFKAIGTAYAVLSNPEKRKQYDQFGDDKSQAARHGHGHGDFHRGFEADISPEDLFNMFFGGGFPSSNVHVYSNGRMRYTYQQRQDRRDNQGDGGLGVFVQLMPILILILVSALSQLMVSSPPYSLSPRPSVGHIHRRVTDHLGVVYYVGDTFSEEYTGSSLKTVERNVEDDYIANLRNNCWKEKQQKEGLLYRARYFGDTDMYHRAQKMGTPSCSRLSEVQASLHG",
# DJB13_HUMAN
"MGQDYYSVLGITRNSEDAQIKQAYRRLALKHHPLKSNEPSSAEIFRQIAEAYDVLSDPMKRGIYDKFGEEGLKGGIPLEFGSQTPWTTGYVFHGKPEKVFHEFFGGNNPFSEFFDAEGSEVDLNFGGLQGRGVKKQDPQVERDLYLSLEDLFFGCTKKIKISRRVLNEDGYSSTIKDKILTIDVKPGWRQGTRITFEKEGDQGPNIIPADIIFIVKEKLHPRFRRENDNLFFVNPIPLGKALTCCTVEVRTLDDRLLNIPINDIIHPKYFKKVPGEGMPLPEDPTKKGDLFIFFDIQFPTRLTPQKKQMLRQALLT",
# DJB14_HUMAN
"MEGNRDEAEKCVEIAREALNAGNREKAQRFLQKAEKLYPLPSARALLEIIMKNGSTAGNSPHCRKPSGSGDQSKPNCTKDSTSGSGEGGKGYTKDQVDGVLSINKCKNYYEVLGVTKDAGDEDLKKAYRKLALKFHPDKNHAPGATDAFKKIGNAYAVLSNPEKRKQYDLTGNEEQACNHQNNGRFNFHRGCEADITPEDLFNIFFGGGFPSGSVHSFSNGRAGYSQQHQHRHSGHEREEERGDGGFSVFIQLMPIIVLILVSLLSQLMVSNPPYSLYPRSGTGQTIKMQTENLGVVYYVNKDFKNEYKGMLLQKVEKSVEEDYVTNIRNNCWKERQQKTDMQYAAKVYRDDRLRRKADALSMDNCKELERLTSLYKGG",
# DNJB2_HUMAN
"MASYYEILDVPRSASADDIKKAYRRKALQWHPDKNPDNKEFAEKKFKEVAEAYEVLSDKHKREIYDRYGREGLTGTGTGPSRAEAGSGGPGFTFTFRSPEEVFREFFGSGDPFAELFDDLGPFSELQNRGSRHSGPFFTFSSSFPGHSDFSSSSFSFSPGAGAFRSVSTSTTFVQGRRITTRRIMENGQERVEVEEDGQLKSVTINGVPDDLALGLELSRREQQPSVTSRSGGTQVQQTPASCPLDSDLSEDEDLQLAMAYSLSEMEAAGKKPAGGREAQHRRQGRPKAQHQDPGLGGTQEGARGEATKRSPSPEEKASRCLIL",
# DNJB3_HUMAN
"MVDYYEVLDVPRQASSEAIKKAYRKLALKWHPDKNPENKEEAERRFKQVAEAYEVLSDAKKRDIYDRYGEAGAEGGCTGGRPFEDPFEYVFSFRDPADVFREFFGGQDPFSFDLLGNPLENILGGSEELLGKQKQSVCTPFLCLQ",
# DNJB4_HUMAN
"MGKDYYCILGIEKGASDEDIKKAYRKQALKFHPDKNKSPQAEEKFKEVAEAYEVLSDPKKREIYDQFGEEGLKGGAGGTDGQGGTFRYTFHGDPHATFAAFFGGSNPFEIFFGRRMGGGRDSEEMEIDGDPFSAFGFSMNGYPRDRNSVGPSRLKQDPPVIHELRVSLEEIYSGCTKRMKISRKRLNADGRSYRSEDKILTIEIKKGWKEGTKITFPREGDETPNSIPADIVFIIKDKDHPKFKRDGSNIIYTAKISLREALCGCSINVPTLDGRNIPMSVNDIVKPGMRRRIIGYGLPFPKNPDQRGDLLIEFEVSFPDTISSSSKEVLRKHLPAS",
# DNJB5_HUMAN
"MGKDYYKILGIPSGANEDEIKKAYRKMALKYHPDKNKEPNAEEKFKEIAEAYDVLSDPKKRGLYDQYGEEGLKTGGGTSGGSSGSFHYTFHGDPHATFASFFGGSNPFDIFFASSRSTRPFSGFDPDDMDVDEDEDPFGAFGRFGFNGLSRGPRRAPEPLYPRRKVQDPPVVHELRVSLEEIYHGSTKRMKITRRRLNPDGRTVRTEDKILHIVIKRGWKEGTKITFPKEGDATPDNIPADIVFVLKDKPHAHFRRDGTNVLYSALISLKEALCGCTVNIPTIDGRVIPLPCNDVIKPGTVKRLRGEGLPFPKVPTQRGDLIVEFKVRFPDRLTPQTRQILKQHLPCS",
# DNJB6_HUMAN
"MVDYYEVLGVQRHASPEDIKKAYRKLALKWHPDKNPENKEEAERKFKQVAEAYEVLSDAKKRDIYDKYGKEGLNGGGGGGSHFDSPFEFGFTFRNPDDVFREFFGGRDPFSFDFFEDPFEDFFGNRRGPRGSRSRGTGSFFSAFSGFPSFGSGFSSFDTGFTSFGSLGHGGLTSFSSTSFGGSGMGNFKSISTSTKMVNGRKITTKRIVENGQERVEVEEDGQLKSLTINGVADDDALAEERMRRGQNALPAQPAGLRPPKPPRPASLLRHAPHCLSEEEGEQDRPRAPGPWDPLASAAGLKEGGKRKKQKQREESKKKKSTKGNH",
# DNJB7_HUMAN
"MVDYYEVLGLQRYASPEDIKKAYHKVALKWHPDKNPENKEEAERKFKEVAEAYEVLSNDEKRDIYDKYGTEGLNGGGSHFDDECEYGFTFHKPDDVFKEIFHERDPFSFHFFEDSLEDLLNRPGSSYGNRNRDAGYFFSTASEYPIFEKFSSYDTGYTSQGSLGHEGLTSFSSLAFDNSGMDNYISVTTSDKIVNGRNINTKKIIESDQEREAEDNGELTFFLVNSVANEEGFAKECSWRTQSFNNYSPNSHSSKHVSQYTFVDNDEGGISWVTSNRDPPIFSAGVKEGGKRKKKKRKEVQKKSTKRNC",
# DNJB8_HUMAN
"MANYYEVLGVQASASPEDIKKAYRKLALRWHPDKNPDNKEEAEKKFKLVSEAYEVLSDSKKRSLYDRAGCDSWRAGGGASTPYHSPFDTGYTFRNPEDIFREFFGGLDPFSFEFWDSPFNSDRGGRGHGLRGAFSAGFGEFPAFMEAFSSFNMLGCSGGSHTTFSSTSFGGSSSGSSGFKSVMSSTEMINGHKVTTKRIVENGQERVEVEEDGQLKSVTVNGKEQLKWMDSK",
# DNJB9_HUMAN
"MATPQSIFIFAICILMITELILASKSYYDILGVPKSASERQIKKAFHKLAMKYHPDKNKSPDAEAKFREIAEAYETLSDANRRKEYDTLGHSAFTSGKGQRGSGSSFEQSFNFNFDDLFKDFGFFGQNQNTGSKKRFENHFQTRQDGGSSRQRHHFQEFSFGGGLFDDMFEDMEKMFSFSGFDSTNQHTVQTENRFHGSSKHCRTVTQRRGNMVTTYTDCSGQ",
# DNJC1_HUMAN
"MTAPCSQPAQLPGRRQLGLVPFPPPPPRTPLLWLLLLLLAAVAPARGWESGDLELFDLVEEVQLNFYQFLGVQQDASSADIRKAYRKLSLTLHPDKNKDENAETQFRQLVAIYEVLKDDERRQRYDDILINGLPDWRQPVFYYRRVRKMSNAELALLLFIILTVGHYAVVWSIYLEKQLDELLSRKKREKKKKTGSKSVDVSKLGASEKNERLLMKPQWHDLLPCKLGIWFCLTLKALPHLIQDAGQFYAKYKETRLKEKEDALTRTELETLQKQKKVKKPKPEFPVYTPLETTYIQSYDHGTSIEEIEEQMDDWLENRNRTQKKQAPEWTEEDLSQLTRSMVKFPGGTPGRWEKIAHELGRSVTDVTTKAKQLKDSVTCSPGMVRLSELKSTVQNSRPIKTATTLPDDMITQREDAEGVAAEEEQEGDSGEQETGATDARPRRRKPARLLEATAKPEPEEKSRAKRQKDFDIAEQNESSDEESLRKERARSAEEPWTQNQQKLLELALQQYPRGSSDRWDKIARCVPSKSKEDCIARYKLLVELVQKKKQAKS",
# DJC10_HUMAN
"MGVWLNKDDYIRDLKRIILCFLIVYMAILVGTDQDFYSLLGVSKTASSREIRQAFKKLALKLHPDKNPNNPNAHGDFLKINRAYEVLKDEDLRKKYDKYGEKGLEDNQGGQYESWNYYRYDFGIYDDDPEIITLERREFDAAVNSGELWFVNFYSPGCSHCHDLAPTWRDFAKEVDGLLRIGAVNCGDDRMLCRMKGVNSYPSLFIFRSGMAPVKYHGDRSKESLVSFAMQHVRSTVTELWTGNFVNSIQTAFAAGIGWLITFCSKGGDCLTSQTRLRLSGMLDGLVNVGWMDCATQDNLCKSLDITTSTTAYFPPGATLNNKEKNSILFLNSLDAKEIYLEVIHNLPDFELLSANTLEDRLAHHRWLLFFHFGKNENSNDPELKKLKTLLKNDHIQVGRFDCSSAPDICSNLYVFQPSLAVFKGQGTKEYEIHHGKKILYDILAFAKESVNSHVTTLGPQNFPANDKEPWLVDFFAPWCPPCRALLPELRRASNLLYGQLKFGTLDCTVHEGLCNMYNIQAYPTTVVFNQSNIHEYEGHHSAEQILEFIEDLMNPSVVSLTPTTFNELVTQRKHNEVWMVDFYSPWCHPCQVLMPEWKRMARTLTGLINVGSIDCQQYHSFCAQENVQRYPEIRFFPPKSNKAYHYHSYNGWNRDAYSLRIWGLGFLPQVSTDLTPQTFSEKVLQGKNHWVIDFYAPWCGPCQNFAPEFELLARMIKGKVKAGKVDCQAYAQTCQKAGIRAYPTVKFYFYERAKRNFQEEQINTRDAKAIAALISEKLETLRNQGKRNKDEL",
# DJC11_HUMAN
"MATALSEEELDNEDYYSLLNVRREASSEELKAAYRRLCMLYHPDKHRDPELKSQAERLFNLVHQAYEVLSDPQTRAIYDIYGKRGLEMEGWEVVERRRTPAEIREEFERLQREREERRLQQRTNPKGTISVGVDATDLFDRYDEEYEDVSGSSFPQIEINKMHISQSIEAPLTATDTAILSGSLSTQNGNGGGSINFALRRVTSAKGWGELEFGAGDLQGPLFGLKLFRNLTPRCFVTTNCALQFSSRGIRPGLTTVLARNLDKNTVGYLQWRWGIQSAMNTSIVRDTKTSHFTVALQLGIPHSFALISYQHKFQDDDQTRVKGSLKAGFFGTVVEYGAERKISRHSVLGAAVSVGVPQGVSLKVKLNRASQTYFFPIHLTDQLLPSAMFYATVGPLVVYFAMHRLIIKPYLRAQKEKELEKQRESAATDVLQKKQEAESAVRLMQESVRRIIEAEESRMGLIIVNAWYGKFVNDKSRKSEKVKVIDVTVPLQCLVKDSKLILTEASKAGLPGFYDPCVGEEKNLKVLYQFRGVLHQVMVLDSEALRIPKQSHRIDTDG",
# DJC12_HUMAN
"MDAILNYRSEDTEDYYTLLGCDELSSVEQILAEFKVRALECHPDKHPENPKAVETFQKLQKAKEILTNEESRARYDHWRRSQMSMPFQQWEALNDSVKTSMHWVVRGKKDLMLEESDKTHTTKMENEECNEQRERKKEELASTAEKTEQKEPKPLEKSVSPQNSDSSGFADVNGWHLRFRWSKDAPSELLRKFRNYEI",
# DJC13_HUMAN
"MNIIRENKDLACFYTTKHSWRGKYKRVFSVGTHAITTYNPNTLEVTNQWPYGDICSISPVGKGQGTEFNLTFRKGSGKKSETLKFSTEHRTELLTEALRFRTDFSEGKITGRRYNCYKHHWSDSRKPVILEVTPGGFDQINPATNRVLCSYDYRNIEGFVDLSDYQGGFCILYGGFSRLHLFASEQREEIIKSAIDHAGNYIGISLRIRKEPLEFEQYLNLRFGKYSTDESITSLAEFVVQKISPRHSEPVKRVLALTETCLVERDPATYNIATLKPLGEVFALVCDSENPQLFTIEFIKGQVRKYSSTERDSLLASLLDGVRASGNRDVCVKMTPTHKGQRWGLLSMPVDEEVESLHLRFLATPPNGNFADAVFRFNANISYSGVLHAVTQDGLFSENKEKLINNAITALLSQEGDVVASNAELESQFQAVRRLVASKAGFLAFTQLPKFRERLGVKVVKALKRSNNGIIHAAVDMLCALMCPMHDDYDLRQEQLNKASLLSSKKFLENLLEKFNSHVDHGTGALVISSLLDFLTFALCAPYSETTEGQQFDMLLEMVASNGRTLFKLFQHPSMAIIKGAGLVMKAIIEEGDKEIATKMQELALSEGALPRHLHTAMFTISSDQRMLTNRQLSRHLVGLWTADNATATNLLKRILPPGLLAYLESSDLVPEKDADRMHVRDNVKIAMDQYGKFNKVPEWQRLAGKAAKEVEKFAKEKVDLVLMHWRDRMGIAQKENINQKPVVLRKRRQRIKIEANWDLFYYRFGQDHARSNLIWNFKTREELKDTLESEMRAFNIDRELGSANVISWNHHEFEVKYECLAEEIKIGDYYLRLLLEEDENEESGSIKRSYEFFNELYHRFLLTPKVNMKCLCLQALAIVYGRCHEEIGPFTDTRYIIGMLERCTDKLERDRLILFLNKLILNKKNVKDLMDSNGIRILVDLLTLAHLHVSRATVPLQSNVIEAAPDMKRESEKEWYFGNADKERSGPYGFHEMQELWTKGMLNAKTRCWAQGMDGWRPLQSIPQLKWCLLASGQAVLNETDLATLILNMLITMCGYFPSRDQDNAIIRPLPKVKRLLSDSTCLPHIIQLLLTFDPILVEKVAILLYHIMQDNPQLPRLYLSGVFFFIMMYTGSNVLPVARFLKYTHTKQAFKSEETKGQDIFQRSILGHILPEAMVCYLENYEPEKFSEIFLGEFDTPEAIWSSEMRRLMIEKIAAHLADFTPRLQSNTRALYQYCPIPIINYPQLENELFCNIYYLKQLCDTLRFPDWPIKDPVKLLKDTLDAWKKEVEKKPPMMSIDDAYEVLNLPQGQGPHDESKIRKAYFRLAQKYHPDKNPEGRDMFEKVNKAYEFLCTKSAKIVDGPDPENIILILKTQSILFNRHKEDLQPYKYAGYPMLIRTITMETSDDLLFSKESPLLPAATELAFHTVNCSALNAEELRRENGLEVLQEAFSRCVAVLTRASKPSDMSVQVCGYISKCYSVAAQFEECREKITEMPSIIKDLCRVLYFGKSIPRVAALGVECVSSFAVDFWLQTHLFQAGILWYLLGFLFNYDYTLEESGIQKSEETNQQEVANSLAKLSVHALSRLGGYLAEEQATPENPTIRKSLAGMLTPYVARKLAVASVTEILKMLNSNTESPYLIWNNSTRAELLEFLESQQENMIKKGDCDKTYGSEFVYSDHAKELIVGEIFVRVYNEVPTFQLEVPKAFAASLLDYIGSQAQYLHTFMAITHAAKVESEQHGDRLPRVEMALEALRNVIKYNPGSESECIGHFKLIFSLLRVHGAGQVQQLALEVVNIVTSNQDCVNNIAESMVLSSLLALLHSLPSSRQLVLETLYALTSSTKIIKEAMAKGALIYLLDMFCNSTHPQVRAQTAELFAKMTADKLIGPKVRITLMKFLPSVFMDAMRDNPEAAVHIFEGTHENPELIWNDNSRDKVSTTVREMMLEHFKNQQDNPEANWKLPEDFAVVFGEAEGELAVGGVFLRIFIAQPAWVLRKPREFLIALLEKLTELLEKNNPHGETLETLTMATVCLFSAQPQLADQVPPLGHLPKVIQAMNHRNNAIPKSAIRVIHALSENELCVRAMASLETIGPLMNGMKKRADTVGLACEAINRMFQKEQSELVAQALKADLVPYLLKLLEGIGLENLDSPAATKAQIVKALKAMTRSLQYGEQVNEILCRSSVWSAFKDQKHDLFISESQTAGYLTGPGVAGYLTAGTSTSVMSNLPPPVDHEAGDLGYQT",
# DJC14_HUMAN
"MAQKHPGERGLYGAHHSGGASLRTLGPSVDPEIPSFSGLRDSAGTAPNGTRCLTEHSGPKHTQHPNPAHWLDPSHGPPGGPGPPRDAEDPDQSETSSEEESGVDQELSKENETGNQKDGNSFLSIPSACNCQGTPGIPEGPYSEGGNGSSSNFCHHCTSPALGEDELEEEYDDEESLKFPSDFSRVSSGKKPPSRRQRHRFPTKEDTREGGRRDPRSPGRHRLGRKRSQADKRKGLGLWGAEELCQLGQAGFWWLIELLVLVGEYVETCGHLIYACRQLKSSDLDLFRVWMGVWTGRLGGWAQVMFQFLSQGFYCGVGLFTRFLKLLGALLLLALALFLGFLQLGWRFLVGLGDRLGWRDKATWLFSWLDSPALQRCLTLLRDSRPWQRLVRIVQWGWLELPWVKQNINRQGNAPVASGRYCQPEEEVARLLTMAGVPEDELNPFHVLGVEATASDVELKKAYRQLAVMVHPDKNHHPRAEEAFKVLRAAWDIVSNAEKRKEYEMKRMAENELSRSVNEFLSKLQDDLKEAMNTMMCSRCQGKHRRFEMDREPKSARYCAECNRLHPAEEGDFWAESSMLGLKITYFALMDGKVYDITEWAGCQRVGISPDTHRVPYHISFGSRIPGTRGRQRATPDAPPADLQDFLSRIFQVPPGQMPNGNFFAAPQPAPGAAAASKPNSTVPKGEAKPKRRKKVRRPFQR",
# DJC15_HUMAN
"MAARGVIAPVGESLRYAEYLQPSAKRPDADVDQQRLVRSLIAVGLGVAALAFAGRYAFRIWKPLEQVITETAKKISTPSFSSYYKGGFEQKMSRREAGLILGVSPSAGKAKIRTAHRRVMILNHPDKGGSPYVAAKINEAKDLLETTTKH",
# DJC16_HUMAN
"MEVRKLSISWQFLIVLVLILQILSALDFDPYRVLGVSRTASQADIKKAYKKLAREWHPDKNKDPGAEDKFIQISKAYEILSNEEKRSNYDQYGDAGENQGYQKQQQQREYRFRHFHENFYFDESFFHFPFNSERRDSIDEKYLLHFSHYVNEVVPDSFKKPYLIKITSDWCFSCIHIEPVWKEVIQELEELGVGIGVVHAGYERRLAHHLGAHSTPSILGIINGKISFFHNAVVRENLRQFVESLLPGNLVEKVTNKNYVRFLSGWQQENKPHVLLFDQTPIVPLLYKLTAFAYKDYLSFGYVYVGLRGTEEMTRRYNINIYAPTLLVFKEHINRPADVIQARGMKKQIIDDFITRNKYLLAARLTSQKLFHELCPVKRSHRQRKYCVVLLTAETTKLSKPFEAFLSFALANTQDTVRFVHVYSNRQQEFADTLLPDSEAFQGKSAVSILERRNTAGRVVYKTLEDPWIGSESDKFILLGYLDQLRKDPALLSSEAVLPDLTDELAPVFLLRWFYSASDYISDCWDSIFHNNWREMMPLLSLIFSALFILFGTVIVQAFSDSNDERESSPPEKEEAQEKTGKTEPSFTKENSSKIPKKGFVEVTELTDVTYTSNLVRLRPGHMNVVLILSNSTKTSLLQKFALEVYTFTGSSCLHFSFLSLDKHREWLEYLLEFAQDAAPIPNQYDKHFMERDYTGYVLALNGHKKYFCLFKPQKTVEEEEAIGSCSDVDSSLYLGESRGKPSCGLGSRPIKGKLSKLSLWMERLLEGSLQRFYIPSWPELD",
# DJC17_HUMAN
"MAVTKELLQMDLYALLGIEEKAADKEVKKAYRQKALSCHPDKNPDNPRAAELFHQLSQALEVLTDAAARAAYDKVRKAKKQAAERTQKLDEKRKKVKLDLEARERQAQAQESEEEEESRSTRTLEQEIERLREEGSRQLEEQQRLIREQIRQERDQRLRGKAENTEGQGTPKLKLKWKCKKEDESKGGYSKDVLLRLLQKYGEVLNLVLSSKKPGTAVVEFATVKAAELAVQNEVGLVDNPLKISWLEGQPQDAVGRSHSGLSKGSVLSERDYESLVMMRMRQAAERQQLIARMQQEDQEGPPT",
# DJC18_HUMAN
"MAATLGSGERWTEAYIDAVRRNKYPEDTPPESHDPCGCCNCMKAQKEKKSENEWTQTRQGEGNSTYSEEQLLGVQRIKKCRNYYEILGVSRDASDEELKKAYRKLALKFHPDKNCAPGATDAFKAIGNAFAVLSNPDKRLRYDEYGDEQVTFTAPRARPYNYYRDFEADITPEELFNVFFGGHFPTGNIHMFSNVTDDTYYYRRRHRHERTQTQKEEEEEKPQTTYSAFIQLLPVLVIVIISVITQLLATNPPYSLFYKSTLGYTISRETQNLQVPYFVDKNFDKAYRGASLHDLEKTIEKDYIDYIQTSCWKEKQQKSELTNLAGLYRDERLKQKAESLKLENCEKLSKLIGLRRGG",
# TIM14_HUMAN
"MASTVVAVGLTIAAAGFAGRYVLQAMKHMEPQVKQVFQSLPKSAFSGGYYRGGFEPKMTKREAALILGVSPTANKGKIRDAHRRIMLLNHPDKGGSPYIAAKINEAKDLLEGQAKK",
# DNJC2_HUMAN
"MLLLPSAADGRGTAITHALTSASTLCQVEPVGRWFEAFVKRRNRNASASFQELEDKKELSEESEDEELQLEEFPMLKTLDPKDWKNQDHYAVLGLGHVRYKATQRQIKAAHKAMVLKHHPDKRKAAGEPIKEGDNDYFTCITKAYEMLSDPVKRRAFNSVDPTFDNSVPSKSEAKDNFFEVFTPVFERNSRWSNKKNVPKLGDMNSSFEDVDIFYSFWYNFDSWREFSYLDEEEKEKAECRDERRWIEKQNRATRAQRKKEEMNRIRTLVDNAYSCDPRIKKFKEEEKAKKEAEKKAKAEAKRKEQEAKEKQRQAELEAARLAKEKEEEEVRQQALLAKKEKDIQKKAIKKERQKLRNSCKTWNHFSDNEAERVKMMEEVEKLCDRLELASLQCLNETLTSCTKEVGKAALEKQIEEINEQIRKEKEEAEARMRQASKNTEKSTGGGGNGSKNWSEDDLQLLIKAVNLFPAGTNSRWEVIANYMNIHSSSGVKRTAKDVIGKAKSLQKLDPHQKDDINKKAFDKFKKEHGVVPQADNATPSERFEGPYTDFTPWTTEEQKLLEQALKTYPVNTPERWEKIAEAVPGRTKKDCMKRYKELVEMVKAKKAAQEQVLNASRAKK",
# DJC21_HUMAN
"MKCHYEALGVRRDASEEELKKAYRKLALKWHPDKNLDNAAEAAEQFKLIQAAYDVLSDPQERAWYDNHREALLKGGFDGEYQDDSLDLLRYFTVTCYSGYGDDEKGFYTVYRNVFEMIAKEELESVLEEEVDDFPTFGDSQSDYDTVVHPFYAYWQSFCTQKNFAWKEEYDTRQASNRWEKRAMEKENKKIRDKARKEKNELVRQLVAFIRKRDKRVQAHRKLVEEQNAEKARKAEEMRRQQKLKQAKLVEQYREQSWMTMANLEKELQEMEARYEKEFGDGSDENEMEEHELKDEEDGKDSDEAEDAELYDDLYCPACDKSFKTEKAMKNHEKSKKHREMVALLKQQLEEEEENFSRPQIDENPLDDNSEEEMEDAPKQKLSKKQKKKKQKPAQNYDDNFNVNGPGEGVKVDPEDTNLNQDSAKELEDSPQENVSVTEIIKPCDDPKSEAKSVPKPKGKKTKDMKKPVRVPAEPQTMSVLISCTTCHSEFPSRNKLFDHLKATGHARAPSSSSLNSATSSQSKKEKRKNR",
# DJC22_HUMAN
"MAKGLLVTYALWAVGGPAGLHHLYLGRDSHALLWMLTLGGGGLGWLWEFWKLPSFVAQANRAQGQRQSPRGVTPPLSPIRFAAQVIVGIYFGLVALISLSSMVNFYIVALPLAVGLGVLLVAAVGNQTSDFKNTLGSAFLTSPIFYGRPIAILPISVAASITAQRHRRYKALVASEPLSVRLYRLGLAYLAFTGPLAYSALCNTAATLSYVAETFGSFLNWFSFFPLLGRLMEFVLLLPYRIWRLLMGETGFNSSCFQEWAKLYEFVHSFQDEKRQLAYQVLGLSEGATNEEIHRSYQELVKVWHPDHNLDQTEEAQRHFLEIQAAYEVLSQPRKPWGSRR",
# DJC24_HUMAN
"MMAVEQMPKKDWYSILGADPSANISDLKQKYQKLILMYHPDKQSTDVPAGTVEECVQKFIEIDQAWKILGNEETKREYDLQRCEDDLRNVGPVDAQVYLEEMSWNEGDHSFYLSCRCGGKYSVSKDEAEEVSLISCDTCSLIIELLHYN",
# DJC25_HUMAN
"MGAPLLSPGWGAGAAGRRWWMLLAPLLPALLLVRPAGALVEGLYCGTRDCYEVLGVSRSAGKAEIARAYRQLARRYHPDRYRPQPGDEGPGRTPQSAEEAFLLVATAYETLKDEETRKDYDYMLDHPEEYYSHYYHYYSRRLAPKVDVRVVILVSVCAISVFQFFSWWNSYNKAISYLATVPKYRIQATEIAKQQGLLKKAKEKGKNKKSKEEIRDEEENIIKNIIKSKIDIKGGYQKPQICDLLLFQIILAPFHLCSYIVWYCRWIYNFNIKGKEYGEEERLYIIRKSMKMSKSQFDSLEDHQKETFLKRELWIKENYEVYKQEQEEELKKKLANDPRWKRYRRWMKNEGPGRLTFVDD",
# DJC27_HUMAN
"MEANMPKRKEPGRSLRIKVISMGNAEVGKSCIIKRYCEKRFVSKYLATIGIDYGVTKVHVRDREIKVNIFDMAGHPFFYEVRNEFYKDTQGVILVYDVGQKDSFDALDAWLAEMKQELGPHGNMENIIFVVCANKIDCTKHRCVDESEGRLWAESKGFLYFETSAQTGEGINEMFQTFYISIVDLCENGGKRPTTNSSASFTKEQADAIRRIRNSKDSWDMLGVKPGASRDEVNKAYRKLAVLLHPDKCVAPGSEDAFKAVVNARTALLKNIK",
# DJC28_HUMAN
"MNTMYVMMAQILRSHLIKATVIPNRVKMLPYFGIIRNRMMSTHKSKKKIREYYRLLNVEEGCSADEVRESFHKLAKQYHPDSGSNTADSATFIRIEKAYRKVLSHVIEQTNASQSKGEEEEDVEKFKYKTPQHRHYLSFEGIGFGTPTQREKHYRQFRADRAAEQVMEYQKQKLQSQYFPDSVIVKNIRQSKQQKITQAIERLVEDLIQESMAKGDFDNLSGKGKPLKKFSDCSYIDPMTHNLNRILIDNGYQPEWILKQKEISDTIEQLREAILVSRKKLGNPMTPTEKKQWNHVCEQFQENIRKLNKRINDFNLIVPILTRQKVHFDAQKEIVRAQKIYETLIKTKEVTDRNPNNLDQGEGEKTPEIKKGFLNWMNLWKFIKIRSF",
# DNJC3_HUMAN
"MVAPGSVTSRLGSVFPFLLVLVDLQYEGAECGVNADVEKHLELGKKLLAAGQLADALSQFHAAVDGDPDNYIAYYRRATVFLAMGKSKAALPDLTKVIQLKMDFTAARLQRGHLLLKQGKLDEAEDDFKKVLKSNPSENEEKEAQSQLIKSDEMQRLRSQALNAFGSGDYTAAIAFLDKILEVCVWDAELRELRAECFIKEGEPRKAISDLKAASKLKNDNTEAFYKISTLYYQLGDHELSLSEVRECLKLDQDHKRCFAHYKQVKKLNKLIESAEELIRDGRYTDATSKYESVMKTEPSIAEYTVRSKERICHCFSKDEKPVEAIRVCSEVLQMEPDNVNALKDRAEAYLIEEMYDEAIQDYETAQEHNENDQQIREGLEKAQRLLKQSQKRDYYKILGVKRNAKKQEIIKAYRKLALQWHPDNFQNEEEKKKAEKKFIDIAAAKEVLSDPEMRKKFDDGEDPLDAESQQGGGGNPFHRSWNSWQGFNPFSSGGPFRFKFHFN",
# DJC30_HUMAN
"MAAMRWRWWQRLLPWRLLQARGFPQNSAPSLGLGARTYSQGDCSYSRTALYDLLGVPSTATQAQIKAAYYRQCFLYHPDRNSGSAEAAERFTRISQAYVVLGSATLRRKYDRGLLSDEDLRGPGVRPSRTPAPDPGSPRTPPPTSRTHDGSRASPGANRTMFNFDAFYQAHYGEQLERERRLRARREALRKRQEYRSMKGLRWEDTRDTAAIFLIFSIFIIIGFYI",
# DNJC4_HUMAN
"MPPLLPLRLCRLWPRNPPSRLLGAAAGQRSRPSTYYELLGVHPGASTEEVKRAFFSKSKELHPDRDPGNPSLHSRFVELSEAYRVLSREQSRRSYDDQLRSGSPPKSPRTTVHDKSAHQTHSSWTPPNAQYWSQFHSVRPQGPQLRQQQHKQNKQVLGYCLLLMLAGMGLHYIAFRKVKQMHLNFMDEKDRIITAFYNEARARARANRGILQQERQRLGQRQPPPSEPTQGPEIVPRGAGP",
# DNJC5_HUMAN
"MADQRQRSLSTSGESLYHVLGLDKNATSDDIKKSYRKLALKYHPDKNPDNPEAADKFKEINNAHAILTDATKRNIYDKYGSLGLYVAEQFGEENVNTYFVLSSWWAKALFVFCGLLTCCYCCCCLCCCFNCCCGKCKPKAPEGEETEFYVSPEDLEAQLQSDEREATDTPIVIQPASATETTQLTADSHPSYHTDGFN",
# DNJ5B_HUMAN
"MACNIPNQRQRTLSTTGEALYEILGLHKGASNEEIKKTYRKLALKHHPDKNPDDPAATEKFKEINNAHAILTDISKRSIYDKYGSLGLYVAEQFGDENVNTYFMLSSWWAKALFVIVGLLTGCYFCCCLCCCCNCCCGHCRPESSVPEEDFYVSPEDLEEQIKSDMEKDVDFPVFLQPTNANEKTQLIKEGSRSYCTDS",
# DNJ5G_HUMAN
"MSTVKEAAHRLSKSEMSLYAVLDLKKGASPEDFKKSYSHSALLPHPPFEYHLGRKLALRYHPDKNPGNAQAAEIFKEINAAHAILSDSKKRKIYDQHGSLGIYLYDHFGEEGVRYYFILNSCWFKTLVILCTLLTCCCFCCCCCFCCGALKPPPEQDSGRKYQQNVQSQPPRSGAKCDFRSEENSEDDF",
# AUXI_HUMAN
"MKDSENKGASSPDMEPSYGGGLFDMVKGGAGRLFSNLKDNLKDTLKDTSSRVIQSVTSYTKGDLDFTYVTSRIIVMSFPLDNVDIGFRNQVDDIRSFLDSRHLDHYTVYNLSPKSYRTAKFHSRVSECSWPIRQAPSLHNLFAVCRNMYNWLLQNPKNVCVVHCLDGRAASSILVGAMFIFCNLYSTPGPAIRLLYAKRPGIGLSPSHRRYLGYMCDLLADKPYRPHFKPLTIKSITVSPIPFFNKQRNGCRPYCDVLIGETKIYSTCTDFERMKEYRVQDGKIFIPLNITVQGDVVVSMYHLRSTIGSRLQAKVTNTQIFQLQFHTGFIPLDTTVLKFTKPELDACDVPEKYPQLFQVTLDVELQPHDKVIDLTPPWEHYCTKDVNPSILFSSHQEHQDTLALGGQAPIDIPPDNPRHYGQSGFFASLCWQDQKSEKSFCEEDHAALVNQESEQSDDELLTLSSPHGNANGDKPHGVKKPSKKQQEPAAPPPPEDVDLLGLEGSAMSNSFSPPAAPPTNSELLSDLFGGGGAAGPTQAGQSGVEDVFHPSGPASTQSTPRRSATSTSASPTLRVGEGATFDPFGAPSKPSGQDLLGSFLNTSSASSDPFLQPTRSPSPTVHASSTPAVNIQPDVSGGWDWHAKPGGFGMGSKSAATSPTGSSHGTPTHQSKPQTLDPFADLGTLGSSSFASKPTTPTGLGGGFPPLSSPQKASPQPMGGGWQQGGAYNWQQPQPKPQPSMPHSSPQNRPNYNVSFSAMPGGQNERGKGSSNLEGKQKAADFEDLLSGQGFNAHKDKKGPRTIAEMRKEEMAKEMDPEKLKILEWIEGKERNIRALLSTMHTVLWAGETKWKPVGMADLVTPEQVKKVYRKAVLVVHPDKATGQPYEQYAKMIFMELNDAWSEFENQGQKPLY",
# DNJC7_HUMAN
"MAAAAECDVVMAATEPELLDDQEAKREAETFKEQGNAYYAKKDYNEAYNYYTKAIDMCPKNASYYGNRAATLMMLGRFREALGDAQQSVRLDDSFVRGHLREGKCHLSLGNAMAACRSFQRALELDHKNAQAQQEFKNANAVMEYEKIAETDFEKRDFRKVVFCMDRALEFAPACHRFKILKAECLAMLGRYPEAQSVASDILRMDSTNADALYVRGLCLYYEDCIEKAVQFFVQALRMAPDHEKACIACRNAKALKAKKEDGNKAFKEGNYKLAYELYTEALGIDPNNIKTNAKLYCNRGTVNSKLRKLDDAIEDCTNAVKLDDTYIKAYLRRAQCYMDTEQYEEAVRDYEKVYQTEKTKEHKQLLKNAQLELKKSKRKDYYKILGVDKNASEDEIKKAYRKRALMHHPDRHSGASAEVQKEEEKKFKEVGEAFTILSDPKKKTRYDSGQDLDEEGMNMGDFDPNNIFKAFFGGPGGFSFEASGPGNFFFQFG",
# DNJC8_HUMAN
"MAASGESGTSGGGGSTEEAFMTFYSEVKQIEKRDSVLTSKNQIERLTRPGSSYFNLNPFEVLQIDPEVTDEEIKKRFRQLSILVHPDKNQDDADRAQKAFEAVDKAYKLLLDQEQKKRALDVIQAGKEYVEHTVKERKKQLKKEGKPTIVEEDDPELFKQAVYKQTMKLFAELEIKRKEREAKEMHERKRQREEEIEAQEKAKREREWQKNFEESRDGRVDSWRNFQANTKGKKEKKNRTFLRPPKVKMEQRE",
# DNJC9_HUMAN
"MGLLDLCEEVFGTADLYRVLGVRREASDGEVRRGYHKVSLQVHPDRVGEGDKEDATRRFQILGKVYSVLSDREQRAVYDEQGTVDEDSPVLTQDRDWEAYWRLLFKKISLEDIQAFEKTYKGSEEELADIKQAYLDFKGDMDQIMESVLCVQYTEEPRIRNIIQQAIDAGEVPSYNAFVKESKQKMNARKRRAQEEAKEAEMSRKELGLDEGVDSLKAAIQSRQKDRQKEMDNFLAQMEAKYCKSSKGGGKKSALKKEKK",
# DAAF4_HUMAN
"MPLQVSDYSWQQTKTAVFLSLPLKGVCVRDTDVFCTENYLKVNFPPFLFEAFLYAPIDDESSKAKIGNDTIVFTLYKKEAAMWETLSVTGVDKEMMQRIREKSILQAQERAKEATEAKAAAKREDQKYALSVMMKIEEEERKKIEDMKENERIKATKALEAWKEYQRKAEEQKKIQREEKLCQKEKQIKEERKKIKYKSLTRNLASRNLAPKGRNSENIFTEKLKEDSIPAPRSVGSIKINFTPRVFPTALRESQVAEEEEWLHKQAEARRAMNTDIAELCDLKEEEKNPEWLKDKGNKLFATENYLAAINAYNLAIRLNNKMPLLYLNRAACHLKLKNLHKAIEDSSKALELLMPPVTDNANARMKAHVRRGTAFCQLELYVEGLQDYEAALKIDPSNKIVQIDAEKIRNVIQGTELKS",
# EDEM1_HUMAN
"MQWRALVLGLVLLRLGLHGVLWLVFGLGPSMGFYQRFPLSFGFQRLRSPDGPASPTSGPVGRPGGVSGPSWLQPPGTGAAQSPRKAPRRPGPGMCGPANWGYVLGGRGRGPDEYEKRYSGAFPPQLRAQMRDLARGMFVFGYDNYMAHAFPQDELNPIHCRGRGPDRGDPSNLNINDVLGNYSLTLVDALDTLAIMGNSSEFQKAVKLVINTVSFDKDSTVQVFEATIRVLGSLLSAHRIITDSKQPFGDMTIKDYDNELLYMAHDLAVRLLPAFENTKTGIPYPRVNLKTGVPPDTNNETCTAGAGSLLVEFGILSRLLGDSTFEWVARRAVKALWNLRSNDTGLLGNVVNIQTGHWVGKQSGLGAGLDSFYEYLLKSYILFGEKEDLEMFNAAYQSIQNYLRRGREACNEGEGDPPLYVNVNMFSGQLMNTWIDSLQAFFPGLQVLIGDVEDAICLHAFYYAIWKRYGALPERYNWQLQAPDVLFYPLRPELVESTYLLYQATKNPFYLHVGMDILQSLEKYTKVKCGYATLHHVIDKSTEDRMESFFLSETCKYLYLLFDEDNPVHKSGTRYMFTTEGHIVSVDEHLRELPWKEFFSEEGGQDQGGKSVHRPKPHELKVINSSSNCNRVPDERRYSLPLKSIYMRQIDQMVGLI",
# EDEM2_HUMAN
"MPFRLLIPLGLLCALLPQHHGAPGPDGSAPDPAHYRERVKAMFYHAYDSYLENAFPFDELRPLTCDGHDTWGSFSLTLIDALDTLLILGNVSEFQRVVEVLQDSVDFDIDVNASVFETNIRVVGGLLSAHLLSKKAGVEVEAGWPCSGPLLRMAEEAARKLLPAFQTPTGMPYGTVNLLHGVNPGETPVTCTAGIGTFIVEFATLSSLTGDPVFEDVARVALMRLWESRSDIGLVGNHIDVLTGKWVAQDAGIGAGVDSYFEYLVKGAILLQDKKLMAMFLEYNKAIRNYTRFDDWYLWVQMYKGTVSMPVFQSLEAYWPGLQSLIGDIDNAMRTFLNYYTVWKQFGGLPEFYNIPQGYTVEKREGYPLRPELIESAMYLYRATGDPTLLELGRDAVESIEKISKVECGFATIKDLRDHKLDNRMESFFLAETVKYLYLLFDPTNFIHNNGSTFDAVITPYGECILGAGGYIFNTEAHPIDPAALHCCQRLKEEQWEVEDLMREFYSLKRSRSKFQKNTVSSGPWEPPARPGTLFSPENHDQARERKPAKQKVPLLSCPSQPFTSKLALLGQVFLDSS",
# EDEM3_HUMAN
"MSEAGGRGCGSPVPQRARWRLVAATAAFCLVSATSVWTAGAEPMSREEKQKLGNQVLEMFDHAYGNYMEHAYPADELMPLTCRGRVRGQEPSRGDVDDALGKFSLTLIDSLDTLVVLNKTKEFEDAVRKVLRDVNLDNDVVVSVFETNIRVLGGLLGGHSLAIMLKEKGEYMQWYNDELLQMAKQLGYKLLPAFNTTSGLPYPRINLKFGIRKPEARTGTETDTCTACAGTLILEFAALSRFTGATIFEEYARKALDFLWEKRQRSSNLVGVTINIHTGDWVRKDSGVGAGIDSYYEYLLKAYVLLGDDSFLERFNTHYDAIMRYISQPPLLLDVHIHKPMLNARTWMDALLAFFPGLQVLKGDIRPAIETHEMLYQVIKKHNFLPEAFTTDFRVHWAQHPLRPEFAESTYFLYKATGDPYYLEVGKTLIENLNKYARVPCGFAAMKDVRTGSHEDRMDSFFLAEMFKYLYLLFADKEDIIFDIEDYIFTTEAHLLPLWLSTTNQSISKKNTTSEYTELDDSNFDWTCPNTQILFPNDPLYAQSIREPLKNVVDKSCPRGIIRVEESFRSGAKPPLRARDFMATNPEHLEILKKMGVSLIHLKDGRVQLVQHAIQAASSIDAEDGLRFMQEMIELSSQQQKEQQLPPRAVQIVSHPFFGRVVLTAGPAQFGLDLSKHKETRGFVASSKPSNGCSELTNPEAVMGKIALIQRGQCMFAEKARNIQNAGAIGGIVIDDNEGSSSDTAPLFQMAGDGKDTDDIKIPMLFLFSKEGSIILDAIREYEEVEVLLSDKAKDRDPEMENEEQPSSENDSQNQSGEQISSSSQEVDLVDQESSEENSLNSHPESLSLADMDNAASISPSEQTSNPTENHETTNLNGECTDLDNQLQEQSETEEDSNPNVSWGKKVQPIDSILADWNEDIEAFEMMEKDEL",
# ERO1A_HUMAN
"MGRGWGFLFGLLGAVWLLSSGHGEEQPPETAAQRCFCQVSGYLDDCTCDVETIDRFNNYRLFPRLQKLLESDYFRYYKVNLKRPCPFWNDISQCGRRDCAVKPCQSDEVPDGIKSASYKYSEEANNLIEECEQAERLGAVDESLSEETQKAVLQWTKHDDSSDNFCEADDIQSPEAEYVDLLLNPERYTGYKGPDAWKIWNVIYEENCFKPQTIKRPLNPLASGQGTSEENTFYSWLEGLCVEKRAFYRLISGLHASINVHLSARYLLQETWLEKKWGHNITEFQQRFDGILTEGEGPRRLKNLYFLYLIELRALSKVLPFFERPDFQLFTGNKIQDEENKMLLLEILHEIKSFPLHFDENSFFAGDKKEAHKLKEDFRLHFRNISRIMDCVGCFKCRLWGKLQTQGLGTALKILFSEKLIANMPESGPSYEFHLTRQEIVSLFNAFGRISTSVKELENFRNLLQNIH",
# ERO1B_HUMAN
"MSQGVRRAGAGQGVAAAVQLLVTLSFLRSVVEAQVTGVLDDCLCDIDSIDNFNTYKIFPKIKKLQERDYFRYYKVNLKRPCPFWAEDGHCSIKDCHVEPCPESKIPVGIKAGHSNKYLKMANNTKELEDCEQANKLGAINSTLSNQSKEAFIDWARYDDSRDHFCELDDERSPAAQYVDLLLNPERYTGYKGTSAWRVWNSIYEENCFKPRSVYRPLNPLAPSRGEDDGESFYTWLEGLCLEKRVFYKLISGLHASINLHLCANYLLEETWGKPSWGPNIKEFKHRFDPVETKGEGPRRLKNLYFLYLIELRALSKVAPYFERSIVDLYTGNAEEDADTKTLLLNIFQDTKSFPMHFDEKSMFAGDKKGAKSLKEEFRLHFKNISRIMDCVGCDKCRLWGKLQTQGLGTALKILFSEKEIQKLPENSPSKGFQLTRQEIVALLNAFGRLSTSIRDLQNFKVLLQHSR",
# ERP44_HUMAN
"MHPAVFLSLPDLRCSLLLLVTWVFTPVTTEITSLDTENIDEILNNADVALVNFYADWCRFSQMLHPIFEEASDVIKEEFPNENQVVFARVDCDQHSDIAQRYRISKYPTLKLFRNGMMMKREYRGQRSVKALADYIRQQKSDPIQEIRDLAEITTLDRSKRNIIGYFEQKDSDNYRVFERVANILHDDCAFLSAFGDVSKPERYSGDNIIYKPPGHSAPDMVYLGAMTNFDVTYNWIQDKCVPLVREITFENGEELTEEGLPFLILFHMKEDTESLEIFQNEVARQLISEKGTINFLHADCDKFRHPLLHIQKTPADCPVIAIDSFRHMYVFGDFKDVLIPGKLKQFVFDLHSGKLHREFHHGPDPTDTAPGEQAQDVASSPPESSFQKLAPSEYRYTLLRDRDEL",
# FKB10_HUMAN
"MFPAGPPSHSLLRLPLLQLLLLVVQAVGRGLGRASPAGGPLEDVVIERYHIPRACPREVQMGDFVRYHYNGTFEDGKKFDSSYDRNTLVAIVVGVGRLITGMDRGLMGMCVNERRRLIVPPHLGYGSIGLAGLIPPDATLYFDVVLLDVWNKEDTVQVSTLLRPPHCPRMVQDGDFVRYHYNGTLLDGTSFDTSYSKGGTYDTYVGSGWLIKGMDQGLLGMCPGERRKIIIPPFLAYGEKGYGTVIPPQASLVFHVLLIDVHNPKDAVQLETLELPPGCVRRAGAGDFMRYHYNGSLMDGTLFDSSYSRNHTYNTYIGQGYIIPGMDQGLQGACMGERRRITIPPHLAYGENGTGDKIPGSAVLIFNVHVIDFHNPADVVEIRTLSRPSETCNETTKLGDFVRYHYNCSLLDGTQLFTSHDYGAPQEATLGANKVIEGLDTGLQGMCVGERRQLIVPPHLAHGESGARGVPGSAVLLFEVELVSREDGLPTGYLFVWHKDPPANLFEDMDLNKDGEVPPEEFSTFIKAQVSEGKGRLMPGQDPEKTIGDMFQNQDRNQDGKITVDELKLKSDEDEERVHEEL",
# FKB11_HUMAN
"MTLRPSLLPLHLLLLLLLSAAVCRAEAGLETESPVRTLQVETLVEPPEPCAEPAAFGDTLHIHYTGSLVDGRIIDTSLTRDPLVIELGQKQVIPGLEQSLLDMCVGEKRRAIIPSHLAYGKRGFPPSVPADAVVQYDVELIALIRANYWLKLVKGILPLVGMAMVPALLGLIGYHLYRKANRPKVSKKKLKEEKRNKSKKK",
# FKB14_HUMAN
"MRLFLWNAVLTLFVTSLIGALIPEPEVKIEVLQKPFICHRKTKGGDLMLVHYEGYLEKDGSLFHSTHKHNNGQPIWFTLGILEALKGWDQGLKGMCVGEKRKLIIPPALGYGKEGKGKIPPESTLIFNIDLLEIRNGPRSHESFQEMDLNDDWKLSKDEVKAYLKKEFEKHGAVVNESHHDALVEDIFDKEDEDKDGFISAREFTYKHDEL",
# FKB15_HUMAN
"MFGAGDEDDTDFLSPSGGARLASLFGLDQAAAGHGNEFFQYTAPKQPKKGQGTAATGNQATPKTAPATMSTPTILVATAVHAYRYTNGQYVKQGKFGAAVLGNHTAREYRILLYISQQQPVTVARIHVNFELMVRPNNYSTFYDDQRQNWSIMFESEKAAVEFNKQVCIAKCNSTSSLDAVLSQDLIVADGPAVEVGDSLEVAYTGWLFQNHVLGQVFDSTANKDKLLRLKLGSGKVIKGWEDGMLGMKKGGKRLLIVPPACAVGSEGVIGWTQATDSILVFEVEVRRVKFARDSGSDGHSVSSRDSAAPSPIPGADNLSADPVVSPPTSIPFKSGEPALRTKSNSLSEQLAINTSPDAVKAKLISRMAKMGQPMLPILPPQLDSNDSEIEDVNTLQGGGQPVVTPSVQPSLHPAHPALPQMTSQAPQPSVTGLQAPSAALMQVSSLDSHSAVSGNAQSFQPYAGMQAYAYPQASAVTSQLQPVRPLYPAPLSQPPHFQGSGDMASFLMTEARQHNTEIRMAVSKVADKMDHLMTKVEELQKHSAGNSMLIPSMSVTMETSMIMSNIQRIIQENERLKQEILEKSNRIEEQNDKISELIERNQRYVEQSNLMMEKRNNSLQTATENTQARVLHAEQEKAKVTEELAAATAQVSHLQLKMTAHQKKETELQMQLTESLKETDLLRGQLTKVQAKLSELQETSEQAQSKFKSEKQNRKQLELKVTSLEEELTDLRVEKESLEKNLSERKKKSAQERSQAEEEIDEIRKSYQEELDKLRQLLKKTRVSTDQAAAEQLSLVQAELQTQWEAKCEHLLASAKDEHLQQYQEVCAQRDAYQQKLVQLQEKCLALQAQITALTKQNEQHIKELEKNKSQMSGVEAAASDPSEKVKKIMNQVFQSLRREFELEESYNGRTILGTIMNTIKMVTLQLLNQQEQEKEESSSEEEEEKAEERPRRPSQEQSASASSGQPQAPLNRERPESPMVPSEQVVEEAVPLPPQALTTSQDGHRRKGDSEAEALSEIKDGSLPPELSCIPSHRVLGPPTSIPPEPLGPVSMDSECEESLAASPMAAKPDNPSGKVCVREVAPDGPLQESSTRLSLTSDPEEGDPLALGPESPGEPQPPQLKKDDVTSSTGPHKELSSTEAGSTVAGAALRPSHHSQRSSLSGDEEDELFKGATLKALRPKAQPEEEDEDEVSMKGRPPPTPLFGDDDDDDDIDWLG",
# FKB1A_HUMAN
"MGVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPFKFMLGKQEVIRGWEEGVAQMSVGQRAKLTISPDYAYGATGHPGIIPPHATLVFDVELLKLE",
# FKB1B_HUMAN
"MGVEIETISPGDGRTFPKKGQTCVVHYTGMLQNGKKFDSSRDRNKPFKFRIGKQEVIKGFEEGAAQMSLGQRAKLTCTPDVAYGATGHPGVIPPNATLIFDVELLNLE",
# FKBP2_HUMAN
"MRLSWFRVLTVLSICLSAVATATGAEGKRKLQIGVKKRVDHCPIKSRKGDVLHMHYTGKLEDGTEFDSSLPQNQPFVFSLGTGQVIKGWDQGLLGMCEGEKRKLVIPSELGYGERGAPPKIPGGATLVFEVELLKIERRTEL",
# FKBP3_HUMAN
"MAAAVPQRAWTVEQLRSEQLPKKDIIKFLQEHGSDSFLAEHKLLGNIKNVAKTANKDHLVTAYNHLFETKRFKGTESISKVSEQVKNVKLNEDKPKETKSEETLDEGPPKYTKSVLKKGDKTNFPKKGDVVHCWYTGTLQDGTVFDTNIQTSAKKKKNAKPLSFKVGVGKVIRGWDEALLTMSKGEKARLEIEPEWAYGKKGQPDAKIPPNAKLTFEVELVDID",
# FKBP4_HUMAN
"MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEMPMIGDRVFVHYTGWLLDGTKFDSSLDRKDKFSFDLGKGEVIKAWDIAIATMKVGEVCHITCKPEYAYGSAGSPPKIPPNATLVFEVELFEFKGEDLTEEEDGGIIRRIQTRGEGYAKPNEGAIVEVALEGYYKDKLFDQRELRFEIGEGENLDLPYGLERAIQRMEKGEHSIVYLKPSYAFGSVGKEKFQIPPNAELKYELHLKSFEKAKESWEMNSEEKLEQSTIVKERGTVYFKEGKYKQALLQYKKIVSWLEYESSFSNEEAQKAQALRLASHLNLAMCHLKLQAFSAAIESCNKALELDSNNEKGLFRRGEAHLAVNDFELARADFQKVLQLYPNNKAAKTQLAVCQQRIRRQLAREKKLYANMFERLAEEENKAKAEASSGDHPTDTEMKEEQKSNTAGSQSQVETEA",
# FKBP5_HUMAN
"MTTDEGAKNNEESPTATVAEQGEDITSKKDRGVLKIVKRVGNGEETPMIGDKVYVHYKGKLSNGKKFDSSHDRNEPFVFSLGKGQVIKAWDIGVATMKKGEICHLLCKPEYAYGSAGSLPKIPSNATLFFEIELLDFKGEDLFEDGGIIRRTKRKGEGYSNPNEGATVEIHLEGRCGGRMFDCRDVAFTVGEGEDHDIPIGIDKALEKMQREEQCILYLGPRYGFGEAGKPKFGIEPNAELIYEVTLKSFEKAKESWEMDTKEKLEQAAIVKEKGTVYFKGGKYMQAVIQYGKIVSWLEMEYGLSEKESKASESFLLAAFLNLAMCYLKLREYTKAVECCDKALGLDSANEKGLYRRGEAQLLMNEFESAKGDFEKVLEVNPQNKAARLQISMCQKKAKEHNERDRRIYANMFKKFAEQDAKEEANKAMGKKTSEGVTNEKGTDSQAMEEEKPEGHV",
# FKBP6_HUMAN
"MGGSALNQGVLEGDDAPGQSLYERLSQRMLDISGDRGVLKDVIREGAGDLVAPDASVLVKYSGYLEHMDRPFDSNYFRKTPRLMKLGEDITLWGMELGLLSMRRGELARFLFKPNYAYGTLGCPPLIPPNTTVLFEIELLDFLDCAESDKFCALSAEQQDQFPLQKVLKVAATEREFGNYLFRQNRFYDAKVRYKRALLLLRRRSAPPEEQHLVEAAKLPVLLNLSFTYLKLDRPTIALCYGEQALIIDQKNAKALFRCGQACLLLTEYQKARDFLVRAQKEQPFNHDINNELKKLASCYRDYVDKEKEMWHRMFAPCGDGSTAGES",
# FKBP7_HUMAN
"MPKTMHFLFRFIVFFYLWGLFTAQRQKKEESTEEVKIEVLHRPENCSKTSKKGDLLNAHYDGYLAKDGSKFYCSRTQNEGHPKWFVLGVGQVIKGLDIAMTDMCPGEKRKVVIPPSFAYGKEGYAEGKIPPDATLIFEIELYAVTKGPRSIETFKQIDMDNDRQLSKAEINLYLQREFEKDEKPRDKSYQDAVLEDIFKKNDHDGDGFISPKEYNVYQHDEL",
# FKBP8_HUMAN
"MASCAEPSEPSAPLPAGVPPLEDFEVLDGVEDAEGEEEEEEEEEEEDDLSELPPLEDMGQPPAEEAEQPGALAREFLAAMEPEPAPAPAPEEWLDILGNGLLRKKTLVPGPPGSSRPVKGQVVTVHLQTSLENGTRVQEEPELVFTLGDCDVIQALDLSVPLMDVGETAMVTADSKYCYGPQGRSPYIPPHAALCLEVTLKTAVDGPDLEMLTGQERVALANRKRECGNAHYQRADFVLAANSYDLAIKAITSSAKVDMTFEEEAQLLQLKVKCLNNLAASQLKLDHYRAALRSCSLVLEHQPDNIKALFRKGKVLAQQGEYSEAIPILRAALKLEPSNKTIHAELSKLVKKHAAQRSTETALYRKMLGNPSRLPAKCPGKGAWSIPWKWLFGATAVALGGVALSVVIAARN",
# FKBP9_HUMAN
"MAFRGWRPPPPPLLLLLLWVTGQAAPVAGLGSDAELQIERRFVPDECPRTVRSGDFVRYHYVGTFPDGQKFDSSYDRDSTFNVFVGKGQLITGMDQALVGMCVNERRFVKIPPKLAYGNEGVSGVIPPNSVLHFDVLLMDIWNSEDQVQIHTYFKPPSCPRTIQVSDFVRYHYNGTFLDGTLFDSSHNRMKTYDTYVGIGWLIPGMDKGLLGMCVGEKRIITIPPFLAYGEDGDGKDIPGQASLVFDVALLDLHNPKDSISIENKVVPENCERISQSGDFLRYHYNGTLLDGTLFDSSYSRNRTFDTYIGQGYVIPGMDEGLLGVCIGEKRRIVVPPHLGYGEEGRGNIPGSAVLVFDIHVIDFHNPSDSISITSHYKPPDCSVLSKKGDYLKYHYNASLLDGTLLDSTWNLGKTYNIVLGSGQVVLGMDMGLREMCVGEKRTVIIPPHLGYGEAGVDGEVPGSAVLVFDIELLELVAGLPEGYMFIWNGEVSPNLFEEIDKDGNGEVLLEEFSEYIHAQVASGKGKLAPGFDAELIVKNMFTNQDRNGDGKVTAEEFKLKDQEAKHDEL",
# FKBPL_HUMAN
"METPPVNTIGEKDTSQPQQEWEKNLRENLDSVIQIRQQPRDPPTETLELEVSPDPASQILEHTQGAEKLVAELEGDSHKSHGSTSQMPEALQASDLWYCPDGSFVKKIVIRGHGLDKPKLGSCCRVLALGFPFGSGPPEGWTELTMGVGPWREETWGELIEKCLESMCQGEEAELQLPGHSGPPVRLTLASFTQGRDSWELETSEKEALAREERARGTELFRAGNPEGAARCYGRALRLLLTLPPPGPPERTVLHANLAACQLLLGQPQLAAQSCDRVLEREPGHLKALYRRGVAQAALGNLEKATADLKKVLAIDPKNRAAQEELGKVVIQGKNQDAGLAQGLRKMFG",
# GAK_HUMAN
"MSLLQSALDFLAGPGSLGGASGRDQSDFVGQTVELGELRLRVRRVLAEGGFAFVYEAQDVGSGREYALKRLLSNEEEKNRAIIQEVCFMKKLSGHPNIVQFCSAASIGKEESDTGQAEFLLLTELCKGQLVEFLKKMESRGPLSCDTVLKIFYQTCRAVQHMHRQKPPIIHRDLKVENLLLSNQGTIKLCDFGSATTISHYPDYSWSAQRRALVEEEITRNTTPMYRTPEIIDLYSNFPIGEKQDIWALGCILYLLCFRQHPFEDGAKLRIVNGKYSIPPHDTQYTVFHSLIRAMLQVNPEERLSIAEVVHQLQEIAAARNVNPKSPITELLEQNGGYGSATLSRGPPPPVGPAGSGYSGGLALAEYDQPYGGFLDILRGGTERLFTNLKDTSSKVIQSVANYAKGDLDISYITSRIAVMSFPAEGVESALKNNIEDVRLFLDSKHPGHYAVYNLSPRTYRPSRFHNRVSECGWAARRAPHLHTLYNICRNMHAWLRQDHKNVCVVHCMDGRAASAVAVCSFLCFCRLFSTAEAAVYMFSMKRCPPGIWPSHKRYIEYMCDMVAEEPITPHSKPILVRAVVMTPVPLFSKQRSGCRPFCEVYVGDERVASTSQEYDKMRDFKIEDGKAVIPLGVTVQGDVLIVIYHARSTLGGRLQAKMASMKMFQIQFHTGFVPRNATTVKFAKYDLDACDIQEKYPDLFQVNLEVEVEPRDRPSREAPPWENSSMRGLNPKILFSSREEQQDILSKFGKPELPRQPGSTAQYDAGAGSPEAEPTDSDSPPSSSADASRFLHTLDWQEEKEAETGAENASSKESESALMEDRDESEVSDEGGSPISSEGQEPRADPEPPGLAAGLVQQDLVFEVETPAVLPEPVPQEDGVDLLGLHSEVGAGPAVPPQACKAPSSNTDLLSCLLGPPEAASQGPPEDLLSEDPLLLASPAPPLSVQSTPRGGPPAAADPFGPLLPSSGNNSQPCSNPDLFGEFLNSDSVTVPPSFPSAHSAPPPSCSADFLHLGDLPGEPSKMTASSSNPDLLGGWAAWTETAASAVAPTPATEGPLFSPGGQPAPCGSQASWTKSQNPDPFADLGDLSSGLQGSPAGFPPGGFIPKTATTPKGSSSWQTSRPPAQGASWPPQAKPPPKACTQPRPNYASNFSVIGAREERGVRAPSFAQKPKVSENDFEDLLSNQGFSSRSDKKGPKTIAEMRKQDLAKDTDPLKLKLLDWIEGKERNIRALLSTLHTVLWDGESRWTPVGMADLVAPEQVKKHYRRAVLAVHPDKAAGQPYEQHAKMIFMELNDAWSEFENQGSRPLF",
# GRPE1_HUMAN
"MAAQCVRLARRSLPALALSLRPSPRLLCTATKQKNSGQNLEEDMGQSEQKADPPATEKTLLEEKVKLEEQLKETVEKYKRALADTENLRQRSQKLVEEAKLYGIQAFCKDLLEVADVLEKATQCVPKEEIKDDNPHLKNLYEGLVMTEVQIQKVFTKHGLLKLNPVGAKFDPYEHEALFHTPVEGKEPGTVALVSKVGYKLHGRTLRPALVGVVKEA",
# GRPE2_HUMAN
"MAVRSLWAGRLRVQRLLAWSAAWESKGWPLPFSTATQRTAGEDCRSEDPPDELGPPLAERALRVKAVKLEKEVQDLTVRYQRAIADCENIRRRTQRCVEDAKIFGIQSFCKDLVEVADILEKTTECISEESEPEDQKLTLEKVFRGLLLLEAKLKSVFAKHGLEKLTPIGDKYDPHEHELICHVPAGVGVQPGTVALVRQDGYKLHGRTIRLARVEVAVESQRRL",
# HSC20_HUMAN
"MWRGRAGALLRVWGFWPTGVPRRRPLSCDAASQAGSNYPRCWNCGGPWGPGREDRFFCPQCRALQAPDPTRDYFSLMDCNRSFRVDTAKLQHRYQQLQRLVHPDFFSQRSQTEKDFSEKHSTLVNDAYKTLLAPLSRGLYLLKLHGIEIPERTDYEMDRQFLIEIMEINEKLAEAESEAAMKEIESIVKAKQKEFTDNVSSAFEQDDFEEAKEILTKMRYFSNIEEKIKLKKIPL",
# HS90A_HUMAN
"MPEETQTQDQPMEEEEVETFAFQAEIAQLMSLIINTFYSNKEIFLRELISNSSDALDKIRYESLTDPSKLDSGKELHINLIPNKQDRTLTIVDTGIGMTKADLINNLGTIAKSGTKAFMEALQAGADISMIGQFGVGFYSAYLVAEKVTVITKHNDDEQYAWESSAGGSFTVRTDTGEPMGRGTKVILHLKEDQTEYLEERRIKEIVKKHSQFIGYPITLFVEKERDKEVSDDEAEEKEDKEEEKEKEEKESEDKPEIEDVGSDEEEEKKDGDKKKKKKIKEKYIDQEELNKTKPIWTRNPDDITNEEYGEFYKSLTNDWEDHLAVKHFSVEGQLEFRALLFVPRRAPFDLFENRKKKNNIKLYVRRVFIMDNCEELIPEYLNFIRGVVDSEDLPLNISREMLQQSKILKVIRKNLVKKCLELFTELAEDKENYKKFYEQFSKNIKLGIHEDSQNRKKLSELLRYYTSASGDEMVSLKDYCTRMKENQKHIYYITGETKDQVANSAFVERLRKHGLEVIYMIEPIDEYCVQQLKEFEGKTLVSVTKEGLELPEDEEEKKKQEEKKTKFENLCKIMKDILEKKVEKVVVSNRLVTSPCCIVTSTYGWTANMERIMKAQALRDNSTMGYMAAKKHLEINPDHSIIETLRQKAEADKNDKSVKDLVILLYETALLSSGFSLEDPQTHANRIYRMIKLGLGIDEDDPTADDTSAAVTEEMPPLEGDDDTSRMEEVD",
# HS90B_HUMAN
"MPEEVHHGEEEVETFAFQAEIAQLMSLIINTFYSNKEIFLRELISNASDALDKIRYESLTDPSKLDSGKELKIDIIPNPQERTLTLVDTGIGMTKADLINNLGTIAKSGTKAFMEALQAGADISMIGQFGVGFYSAYLVAEKVVVITKHNDDEQYAWESSAGGSFTVRADHGEPIGRGTKVILHLKEDQTEYLEERRVKEVVKKHSQFIGYPITLYLEKEREKEISDDEAEEEKGEKEEEDKDDEEKPKIEDVGSDEEDDSGKDKKKKTKKIKEKYIDQEELNKTKPIWTRNPDDITQEEYGEFYKSLTNDWEDHLAVKHFSVEGQLEFRALLFIPRRAPFDLFENKKKKNNIKLYVRRVFIMDSCDELIPEYLNFIRGVVDSEDLPLNISREMLQQSKILKVIRKNIVKKCLELFSELAEDKENYKKFYEAFSKNLKLGIHEDSTNRRRLSELLRYHTSQSGDEMTSLSEYVSRMKETQKSIYYITGESKEQVANSAFVERVRKRGFEVVYMTEPIDEYCVQQLKEFDGKSLVSVTKEGLELPEDEEEKKKMEESKAKFENLCKLMKEILDKKVEKVTISNRLVSSPCCIVTSTYGWTANMERIMKAQALRDNSTMGYMMAKKHLEINPDHPIVETLRQKAEADKNDKAVKDLVVLLFETALLSSGFSLEDPQTHSNRIYRMIKLGLGIDEDEVAAEEPNAAVPDEIPPLEGDEDASRMEEVD",
# ENPL_HUMAN
"MRALWVLGLCCVLLTFGSVRADDEVDVDGTVEEDLGKSREGSRTDDEVVQREEEAIQLDGLNASQIRELREKSEKFAFQAEVNRMMKLIINSLYKNKEIFLRELISNASDALDKIRLISLTDENALSGNEELTVKIKCDKEKNLLHVTDTGVGMTREELVKNLGTIAKSGTSEFLNKMTEAQEDGQSTSELIGQFGVGFYSAFLVADKVIVTSKHNNDTQHIWESDSNEFSVIADPRGNTLGRGTTITLVLKEEASDYLELDTIKNLVKKYSQFINFPIYVWSSKTETVEEPMEEEEAAKEEKEESDDEAAVEEEEEEKKPKTKKVEKTVWDWELMNDIKPIWQRPSKEVEEDEYKAFYKSFSKESDDPMAYIHFTAEGEVTFKSILFVPTSAPRGLFDEYGSKKSDYIKLYVRRVFITDDFHDMMPKYLNFVKGVVDSDDLPLNVSRETLQQHKLLKVIRKKLVRKTLDMIKKIADDKYNDTFWKEFGTNIKLGVIEDHSNRTRLAKLLRFQSSHHPTDITSLDQYVERMKEKQDKIYFMAGSSRKEAESSPFVERLLKKGYEVIYLTEPVDEYCIQALPEFDGKRFQNVAKEGVKFDESEKTKESREAVEKEFEPLLNWMKDKALKDKIEKAVVSQRLTESPCALVASQYGWSGNMERIMKAQAYQTGKDISTNYYASQKKTFEINPRHPLIRDMLRRIKEDEDDKTVLDLAVVLFETATLRSGYLLPDTKAYGDRIERMLRLSLNIDPDAKVEEEPEEEPEETAEDTTEDTEQDEDEEMDVGTDEEEETAKESTAEKDEL",
# HS12A_HUMAN
"MADKEAGGSDGPRETAPTSAYSSPARSLGDTGITPLSPSHIVNDTDSNVSEQQSFLVVVAVDFGTTSSGYAYSFTKEPECIHVMRRWEGGDPGVSNQKTPTTILLTPERKFHSFGYAARDFYHDLDPNEAKQWLYLEKFKMKLHTTGDLTMDTDLTAANGKKVKALEIFAYALQYFKEQALKELSDQAGSEFENSDVRWVITVPAIWKQPAKQFMRQAAYQAGLASPENSEQLIIALEPEAASIYCRKLRLHQMIELSSKAAVNGYSGSDTVGAGFTQAKEHIRRNRQSRTFLVENVIGEIWSELEEGDKYVVVDSGGGTVDLTVHQIRLPEGHLKELYKATGGPYGSLGVDYEFEKLLYKIFGEDFIEQFKIKRPAAWVDLMIAFESRKRAAAPDRTNPLNITLPFSFIDYYKKFRGHSVEHALRKSNVDFVKWSSQGMLRMSPDAMNALFKPTIDSIIEHLRDLFQKPEVSTVKFLFLVGGFAEAPLLQQAVQAAFGDQCRIIIPQDVGLTILKGAVLFGLDPAVIKVRRSPLTYGVGVLNRYVEGKHPPEKLLVKDGTRWCTDVFDKFISADQSVALGELVKRSYTPAKPSQLVIVINIYSSEHDNVSFITDPGVKKCGTLRLDLTGTSGTAVPARREIQTLMQFGDTEIKATAIDIATSKSVKVGIDFLNY",
# HS12B_HUMAN
"MLAVPEMGLQGLYIGSSPERSPVPSPPGSPRTQESCGIAPLTPSQSPKPEVRAPQQASFSVVVAIDFGTTSSGYAFSFASDPEAIHMMRKWEGGDPGVAHQKTPTCLLLTPEGAFHSFGYTARDYYHDLDPEEARDWLYFEKFKMKIHSATDLTLKTQLEAVNGKTMPALEVFAHALRFFREHALQELREQSPSLPEKDTVRWVLTVPAIWKQPAKQFMREAAYLAGLVSRENAEQLLIALEPEAASVYCRKLRLHQLLDLSGRAPGGGRLGERRSIDSSFRQAREQLRRSRHSRTFLVESGVGELWAEMQAGDRYVVADCGGGTVDLTVHQLEQPHGTLKELYKASGGPYGAVGVDLAFEQLLCRIFGEDFIATFKRQRPAAWVDLTIAFEARKRTAGPHRAGALNISLPFSFIDFYRKQRGHNVETALRRSSVNFVKWSSQGMLRMSCEAMNELFQPTVSGIIQHIEALLARPEVQGVKLLFLVGGFAESAVLQHAVQAALGARGLRVVVPHDVGLTILKGAVLFGQAPGVVRVRRSPLTYGVGVLNRFVPGRHPPEKLLVRDGRRWCTDVFERFVAAEQSVALGEEVRRSYCPARPGQRRVLINLYCCAAEDARFITDPGVRKCGALSLELEPADCGQDTAGAPPGRREIRAAMQFGDTEIKVTAVDVSTNRSVRASIDFLSN",
# HSP13_HUMAN
"MAREMTILGSAVLTLLLAGYLAQQYLPLPTPKVIGIDLGTTYCSVGVFFPGTGKVKVIPDENGHISIPSMVSFTDNDVYVGYESVELADSNPQNTIYDAKRFIGKIFTAEELEAEIGRYPFKVLNKNGMVEFSVTSNETITVSPEYVGSRLLLKLKEMAEAYLGMPVANAVISVPAEFDLKQRNSTIEAANLAGLKILRVINEPTAAAMAYGLHKADVFHVLVIDLGGGTLDVSLLNKQGGMFLTRAMSGNNKLGGQDFNQRLLQYLYKQIYQTYGFVPSRKEEIHRLRQAVEMVKLNLTLHQSAQLSVLLTVEEQDRKEPHSSDTELPKDKLSSADDHRVNSGFGRGLSDKKSGESQVLFETEISRKLFDTLNEDLFQKILVPIQQVLKEGHLEKTEIDEVVLVGGSTRIPRIRQVIQEFFGKDPNTSVDPDLAVVTGVAIQAGIDGGSWPLQVSALEIPNKHLQKTNFN",
# HSP7E_HUMAN
"MAAIGVHLGCTSACVAVYKDGRAGVVANDAGDRVTPAVVAYSENEEIVGLAAKQSRIRNISNTVMKVKQILGRSSSDPQAQKYIAESKCLVIEKNGKLRYEIDTGEETKFVNPEDVARLIFSKMKETAHSVLGSDANDVVITVPFDFGEKQKNALGEAARAAGFNVLRLIHEPSAALLAYGIGQDSPTGKSNILVFKLGGTSLSLSVMEVNSGIYRVLSTNTDDNIGGAHFTETLAQYLASEFQRSFKHDVRGNARAMMKLTNSAEVAKHSLSTLGSANCFLDSLYEGQDFDCNVSRARFELLCSPLFNKCIEAIRGLLDQNGFTADDINKVVLCGGSSRIPKLQQLIKDLFPAVELLNSIPPDEVIPIGAAIEAGILIGKENLLVEDSLMIECSARDILVKGVDESGASRFTVLFPSGTPLPARRQHTLQAPGSISSVCLELYESDGKNSAKEETKFAQVVLQDLDKKENGLRDILAVLTMKRDGSLHVTCTDQETGKCEAISIEIAS",
# HS71A_HUMAN
"MAKAAAIGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVALNPQNTVFDAKRLIGRKFGDPVVQSDMKHWPFQVINDGDKPKVQVSYKGETKAFYPEEISSMVLTKMKEIAEAYLGYPVTNAVITVPAYFNDSQRQATKDAGVIAGLNVLRIINEPTAAAIAYGLDRTGKGERNVLIFDLGGGTFDVSILTIDDGIFEVKATAGDTHLGGEDFDNRLVNHFVEEFKRKHKKDISQNKRAVRRLRTACERAKRTLSSSTQASLEIDSLFEGIDFYTSITRARFEELCSDLFRSTLEPVEKALRDAKLDKAQIHDLVLVGGSTRIPKVQKLLQDFFNGRDLNKSINPDEAVAYGAAVQAAILMGDKSENVQDLLLLDVAPLSLGLETAGGVMTALIKRNSTIPTKQTQIFTTYSDNQPGVLIQVYEGERAMTKDNNLLGRFELSGIPPAPRGVPQIEVTFDIDANGILNVTATDKSTGKANKITITNDKGRLSKEEIERMVQEAEKYKAEDEVQRERVSAKNALESYAFNMKSAVEDEGLKGKISEADKKKVLDKCQEVISWLDANTLAEKDEFEHKRKELEQVCNPIISGLYQGAGGPGPGGFGAQGPKGGSGSGPTIEEVD",
# HS71B_HUMAN
"MAKAAAIGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVALNPQNTVFDAKRLIGRKFGDPVVQSDMKHWPFQVINDGDKPKVQVSYKGETKAFYPEEISSMVLTKMKEIAEAYLGYPVTNAVITVPAYFNDSQRQATKDAGVIAGLNVLRIINEPTAAAIAYGLDRTGKGERNVLIFDLGGGTFDVSILTIDDGIFEVKATAGDTHLGGEDFDNRLVNHFVEEFKRKHKKDISQNKRAVRRLRTACERAKRTLSSSTQASLEIDSLFEGIDFYTSITRARFEELCSDLFRSTLEPVEKALRDAKLDKAQIHDLVLVGGSTRIPKVQKLLQDFFNGRDLNKSINPDEAVAYGAAVQAAILMGDKSENVQDLLLLDVAPLSLGLETAGGVMTALIKRNSTIPTKQTQIFTTYSDNQPGVLIQVYEGERAMTKDNNLLGRFELSGIPPAPRGVPQIEVTFDIDANGILNVTATDKSTGKANKITITNDKGRLSKEEIERMVQEAEKYKAEDEVQRERVSAKNALESYAFNMKSAVEDEGLKGKISEADKKKVLDKCQEVISWLDANTLAEKDEFEHKRKELEQVCNPIISGLYQGAGGPGPGGFGAQGPKGGSGSGPTIEEVD",
# HS71L_HUMAN
"MATAKGIAIGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVAMNPQNTVFDAKRLIGRKFNDPVVQADMKLWPFQVINEGGKPKVLVSYKGENKAFYPEEISSMVLTKLKETAEAFLGHPVTNAVITVPAYFNDSQRQATKDAGVIAGLNVLRIINEPTAAAIAYGLDKGGQGERHVLIFDLGGGTFDVSILTIDDGIFEVKATAGDTHLGGEDFDNRLVSHFVEEFKRKHKKDISQNKRAVRRLRTACERAKRTLSSSTQANLEIDSLYEGIDFYTSITRARFEELCADLFRGTLEPVEKALRDAKMDKAKIHDIVLVGGSTRIPKVQRLLQDYFNGRDLNKSINPDEAVAYGAAVQAAILMGDKSEKVQDLLLLDVAPLSLGLETAGGVMTALIKRNSTIPTKQTQIFTTYSDNQPGVLIQVYEGERAMTKDNNLLGRFDLTGIPPAPRGVPQIEVTFDIDANGILNVTATDKSTGKVNKITITNDKGRLSKEEIERMVLDAEKYKAEDEVQREKIAAKNALESYAFNMKSVVSDEGLKGKISESDKNKILDKCNELLSWLEVNQLAEKDEFDHKRKELEQMCNPIITKLYQGGCTGPACGTGYVPGRPATGPTIEEVD",
# HSP72_HUMAN
"MSARGPAIGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVAMNPTNTIFDAKRLIGRKFEDATVQSDMKHWPFRVVSEGGKPKVQVEYKGETKTFFPEEISSMVLTKMKEIAEAYLGGKVHSAVITVPAYFNDSQRQATKDAGTITGLNVLRIINEPTAAAIAYGLDKKGCAGGEKNVLIFDLGGGTFDVSILTIEDGIFEVKSTAGDTHLGGEDFDNRMVSHLAEEFKRKHKKDIGPNKRAVRRLRTACERAKRTLSSSTQASIEIDSLYEGVDFYTSITRARFEELNADLFRGTLEPVEKALRDAKLDKGQIQEIVLVGGSTRIPKIQKLLQDFFNGKELNKSINPDEAVAYGAAVQAAILIGDKSENVQDLLLLDVTPLSLGIETAGGVMTPLIKRNTTIPTKQTQTFTTYSDNQSSVLVQVYEGERAMTKDNNLLGKFDLTGIPPAPRGVPQIEVTFDIDANGILNVTAADKSTGKENKITITNDKGRLSKDDIDRMVQEAERYKSEDEANRDRVAAKNALESYTYNIKQTVEDEKLRGKISEQDKNKILDKCQEVINWLDRNQMAEKDEYEHKQKELERVCNPIISKLYQGGPGGGSGGGGSGASGGPTIEEVD",
# HSP74_HUMAN
"MSVVGIDLGFQSCYVAVARAGGIETIANEYSDRCTPACISFGPKNRSIGAAAKSQVISNAKNTVQGFKRFHGRAFSDPFVEAEKSNLAYDIVQLPTGLTGIKVTYMEEERNFTTEQVTAMLLSKLKETAESVLKKPVVDCVVSVPCFYTDAERRSVMDATQIAGLNCLRLMNETTAVALAYGIYKQDLPALEEKPRNVVFVDMGHSAYQVSVCAFNRGKLKVLATAFDTTLGGRKFDEVLVNHFCEEFGKKYKLDIKSKIRALLRLSQECEKLKKLMSANASDLPLSIECFMNDVDVSGTMNRGKFLEMCNDLLARVEPPLRSVLEQTKLKKEDIYAVEIVGGATRIPAVKEKISKFFGKELSTTLNADEAVTRGCALQCAILSPAFKVREFSITDVVPYPISLRWNSPAEEGSSDCEVFSKNHAAPFSKVLTFYRKEPFTLEAYYSSPQDLPYPDPAIAQFSVQKVTPQSDGSSSKVKVKVRVNVHGIFSVSSASLVEVHKSEENEEPMETDQNAKEEEKMQVDQEEPHVEEQQQQTPAENKAESEEMETSQAGSKDKKMDQPPQAKKAKVKTSTVDLPIENQLLWQIDREMLNLYIENEGKMIMQDKLEKERNDAKNAVEEYVYEMRDKLSGEYEKFVSEDDRNSFTLKLEDTENWLYEDGEDQPKQVYVDKLAELKNLGQPIKIRFQESEERPKLFEELGKQIQQYMKIISSFKNKEDQYDHLDAADMTKVEKSTNEAMEWMNNKLNLQNKQSLTMDPVVKSKEIEAKIKELTSTCSPIISKPKPKVEPPKEEQKNAEQNGPVDGQGDNPGPQAAEQGTDTAVPSDSDKKLPEMDID",
# HS74L_HUMAN
"MSVVGIDLGFLNCYIAVARSGGIETIANEYSDRCTPACISLGSRTRAIGNAAKSQIVTNVRNTIHGFKKLHGRSFDDPIVQTERIRLPYELQKMPNGSAGVKVRYLEEERPFAIEQVTGMLLAKLKETSENALKKPVADCVISIPSFFTDAERRSVMAAAQVAGLNCLRLMNETTAVALAYGIYKQDLPPLDEKPRNVVFIDMGHSAYQVLVCAFNKGKLKVLATTFDPYLGGRNFDEALVDYFCDEFKTKYKINVKENSRALLRLYQECEKLKKLMSANASDLPLNIECFMNDLDVSSKMNRAQFEQLCASLLARVEPPLKAVMEQANLQREDISSIEIVGGATRIPAVKEQITKFFLKDISTTLNADEAVARGCALQCAILSPAFKVREFSITDLVPYSITLRWKTSFEDGSGECEVFCKNHPAPFSKVITFHKKEPFELEAFYTNLHEVPYPDARIGSFTIQNVFPQSDGDSSKVKVKVRVNIHGIFSVASASVIEKQNLEGDHSDAPMETETSFKNENKDNMDKMQVDQEEGHQKCHAEHTPEEEIDHTGAKTKSAVSDKQDRLNQTLKKGKVKSIDLPIQSSLCRQLGQDLLNSYIENEGKMIMQDKLEKERNDAKNAVEEYVYDFRDRLGTVYEKFITPEDLSKLSAVLEDTENWLYEDGEDQPKQVYVDKLQELKKYGQPIQMKYMEHEERPKALNDLGKKIQLVMKVIEAYRNKDERYDHLDPTEMEKVEKCISDAMSWLNSKMNAQNKLSLTQDPVVKVSEIVAKSKELDNFCNPIIYKPKPKAEVPEDKPKANSEHNGPMDGQSGTETKSDSTKDSSQHTKSSGEMEVD",
# BIP_HUMAN
"MKLSLVAAMLLLLSAARAEEEDKKEDVGTVVGIDLGTTYSCVGVFKNGRVEIIANDQGNRITPSYVAFTPEGERLIGDAAKNQLTSNPENTVFDAKRLIGRTWNDPSVQQDIKFLPFKVVEKKTKPYIQVDIGGGQTKTFAPEEISAMVLTKMKETAEAYLGKKVTHAVVTVPAYFNDAQRQATKDAGTIAGLNVMRIINEPTAAAIAYGLDKREGEKNILVFDLGGGTFDVSLLTIDNGVFEVVATNGDTHLGGEDFDQRVMEHFIKLYKKKTGKDVRKDNRAVQKLRREVEKAKRALSSQHQARIEIESFYEGEDFSETLTRAKFEELNMDLFRSTMKPVQKVLEDSDLKKSDIDEIVLVGGSTRIPKIQQLVKEFFNGKEPSRGINPDEAVAYGAAVQAGVLSGDQDTGDLVLLDVCPLTLGIETVGGVMTKLIPRNTVVPTKKSQIFSTASDNQPTVTIKVYEGERPLTKDNHLLGTFDLTGIPPAPRGVPQIEVTFEIDVNGILRVTAEDKGTGNKNKITITNDQNRLTPEEIERMVNDAEKFAEEDKKLKERIDTRNELESYAYSLKNQIGDKEKLGGKLSSEDKETMEKAVEEKIEWLESHQDADIEDFKAKKKELEEIVQPIISKLYGSAGPPPTGEEDTAEKDEL",
# HSP76_HUMAN
"MQAPRELAVGIDLGTTYSCVGVFQQGRVEILANDQGNRTTPSYVAFTDTERLVGDAAKSQAALNPHNTVFDAKRLIGRKFADTTVQSDMKHWPFRVVSEGGKPKVRVCYRGEDKTFYPEEISSMVLSKMKETAEAYLGQPVKHAVITVPAYFNDSQRQATKDAGAIAGLNVLRIINEPTAAAIAYGLDRRGAGERNVLIFDLGGGTFDVSVLSIDAGVFEVKATAGDTHLGGEDFDNRLVNHFMEEFRRKHGKDLSGNKRALRRLRTACERAKRTLSSSTQATLEIDSLFEGVDFYTSITRARFEELCSDLFRSTLEPVEKALRDAKLDKAQIHDVVLVGGSTRIPKVQKLLQDFFNGKELNKSINPDEAVAYGAAVQAAVLMGDKCEKVQDLLLLDVAPLSLGLETAGGVMTTLIQRNATIPTKQTQTFTTYSDNQPGVFIQVYEGERAMTKDNNLLGRFELSGIPPAPRGVPQIEVTFDIDANGILSVTATDRSTGKANKITITNDKGRLSKEEVERMVHEAEQYKAEDEAQRDRVAAKNSLEAHVFHVKGSLQEESLRDKIPEEDRRKMQDKCREVLAWLEHNQLAEKEEYEHQKRELEQICRPIFSRLYGGPGVPGGSSCGTQARQGDPSTGPIIEEVD",
# HSP77_HUMAN
"MQAPRELAVGIDLGTTYSCVGVFQQGRVEILANDQGNRTTPSYVAFTDTERLVGDAAKSQAALNPHNTVFDAKRLIGRKFADTTVQSDMKHWPFQVVSEGGKPKVRVCYRGEDKTFYPEEISSMVLSKMKETAEAYLGQPVKHAVITVPTYFSNSQRQATKDAGAIAGLKVLPIINEATAAAIAYGLDRRGAGKRNVLIFDLGGGTFDVSVLSIDAGVFEVKATAGDTHLGGEDFDNRLVNHFMEEFRRKHGKDLSGNKRALRRLRTACERAKRTPSSSTQATLEIDSLFEGVDFYKSITRARFEELCSDLFRSTLEPVEKALRDAKLDKAQIHDFVLGGGLHSHPQGAEVAAGLLQRQGAEQEHQP",
# HSP7C_HUMAN
"MSKGPAVGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVAMNPTNTVFDAKRLIGRRFDDAVVQSDMKHWPFMVVNDAGRPKVQVEYKGETKSFYPEEVSSMVLTKMKEIAEAYLGKTVTNAVVTVPAYFNDSQRQATKDAGTIAGLNVLRIINEPTAAAIAYGLDKKVGAERNVLIFDLGGGTFDVSILTIEDGIFEVKSTAGDTHLGGEDFDNRMVNHFIAEFKRKHKKDISENKRAVRRLRTACERAKRTLSSSTQASIEIDSLYEGIDFYTSITRARFEELNADLFRGTLDPVEKALRDAKLDKSQIHDIVLVGGSTRIPKIQKLLQDFFNGKELNKSINPDEAVAYGAAVQAAILSGDKSENVQDLLLLDVTPLSLGIETAGGVMTVLIKRNTTIPTKQTQTFTTYSDNQPGVLIQVYEGERAMTKDNNLLGKFELTGIPPAPRGVPQIEVTFDIDANGILNVSAVDKSTGKENKITITNDKGRLSKEDIERMVQEAEKYKAEDEKQRDKVSSKNSLESYAFNMKATVEDEKLQGKINDEDKQKILDKCNEIINWLDKNQTAEKEEFEHQQKELEKVCNPIITKLYQSAGGMPGGMPGGFPGGGAPPSGGASSGPTIEEVD",
# HSPA9_HUMAN
"MISASRAAAARLVGAAASRGPTAARHQDSWNGLSHEAFRLVSRRDYASEAIKGAVVGIDLGTTNSCVAVMEGKQAKVLENAEGARTTPSVVAFTADGERLVGMPAKRQAVTNPNNTFYATKRLIGRRYDDPEVQKDIKNVPFKIVRASNGDAWVEAHGKLYSPSQIGAFVLMKMKETAENYLGHTAKNAVITVPAYFNDSQRQATKDAGQISGLNVLRVINEPTAAALAYGLDKSEDKVIAVYDLGGGTFDISILEIQKGVFEVKSTNGDTFLGGEDFDQALLRHIVKEFKRETGVDLTKDNMALQRVREAAEKAKCELSSSVQTDINLPYLTMDSSGPKHLNMKLTRAQFEGIVTDLIRRTIAPCQKAMQDAEVSKSDIGEVILVGGMTRMPKVQQTVQDLFGRAPSKAVNPDEAVAIGAAIQGGVLAGDVTDVLLLDVTPLSLGIETLGGVFTKLINRNTTIPTKKSQVFSTAADGQTQVEIKVCQGEREMAGDNKLLGQFTLIGIPPAPRGVPQIEVTFDIDANGIVHVSAKDKGTGREQQIVIQSSGGLSKDDIENMVKNAEKYAEEDRRKKERVEAVNMAEGIIHDTETKMEEFKDQLPADECNKLKEEISKMRELLARKDSETGENIRQAASSLQQASLKLFEMAYKKMASEREGSGSSGTGEQKEDQKEEKQ",
# HSPB1_HUMAN
"MTERRVPFSLLRGPSWDPFRDWYPHSRLFDQAFGLPRLPEEWSQWLGGSSWPGYVRPLPPAAIESPAVAAPAYSRALSRQLSSGVSEIRHTADRWRVSLDVNHFAPDELTVKTKDGVVEITGKHEERQDEHGYISRCFTRKYTLPPGVDPTQVSSSLSPEGTLTVEAPMPKLATQSNEITIPVTFESRAQLGGPEAAKSDETAAK",
# IFT25_HUMAN
"MRKIDLCLSSEGSEVILATSSDEKHPPENIIDGNPETFWTTTGMFPQEFIICFHKHVRIERLVIQSYFVQTLKIEKSTSKEPVDFEQWIEKDLVHTEGQLQNEEIVAHDGSATYLRFIIVSAFDHFASVHSVSAEGTVVSNLSS",
# HSPB2_HUMAN
"MSGRSVPHAHPATAEYEFANPSRLGEQRFGEGLLPEEILTPTLYHGYYVRPRAAPAGEGSRAGASELRLSEGKFQAFLDVSHFTPDEVTVRTVDNLLEVSARHPQRLDRHGFVSREFCRTYVLPADVDPWRVRAALSHDGILNLEAPRGGRHLDTEVNEVYISLLPAPPDPEEEEEAAIVEP",
# HSPB3_HUMAN
"MAKIILRHLIEIPVRYQEEFEARGLEDCRLDHALYALPGPTIVDLRKTRAAQSPPVDSAAETPPREGKSHFQILLDVVQFLPEDIIIQTFEGWLLIKAQHGTRMDEHGFISRSFTRQYKLPDGVEIKDLSAVLCHDGILVVEVKDPVGTK",
# HSPB6_HUMAN
"MEIPVPVQPSWLRRASAPLPGLSAPGRLFDQRFGEGLLEAELAALCPTTLAPYYLRAPSVALPVAQVPTDPGHFSVLLDVKHFSPEEIAVKVVGEHVEVHARHEERPDEHGFVAREFHRRYRLPPGVDPAAVTSALSPEGVLSIQAAPASAQAPPPAAAK",
# HSPB7_HUMAN
"MSHRTSSTFRAERSFHSSSSSSSSSTSSSASRALPAQDPPMEKALSMFSDDFGSFMRPHSEPLAFPARPGGAGNIKTLGDAYEFAVDVRDFSPEDIIVTTSNNHIEVRAEKLAADGTVMNTFAHKCQLPEDVDPTSVTSALREDGSLTIRARRHPHTEHVQQTFRTEIKI",
# HSPB8_HUMAN
"MADGQMPFSCHYPSRLRRDPFRDSPLSSRLLDDGFGMDPFPDDLTASWPDWALPRLSSAWPGTLRSGMVPRGPTATARFGVPAEGRTPPPFPGEPWKVCVNVHSFKPEELMVKTKDGYVEVSGKHEEKQQEGGIVSKNFTKKIQLPAEVDPVTVFASLSPEGLLIIEAPQVPPYSTFGESSFNNELPQDSQEVTCT",
# HSPB9_HUMAN
"MQRVGNTFSNESRVASRCPSVGLAERNRVATMPVRLLRDSPAAQEDNDHARDGFQMKLDAHGFAPEELVVQVDGQWLMVTGQQQLDVRDPERVSYRMSQKVHRKMLPSNLSPTAMTCCLTPSGQLWVRGQCVALALPEAQTGPSPRLGSLGSKASNLTR",
# HPBP1_HUMAN
"MSDEGSRGSRLPLALPPASQGCSSGGGGGGSSAGGSGNSRPPRNLQGLLQMAITAGSEEPDPPPEPMSEERRQWLQEAMSAAFRGQREEVEQMKSCLRVLSQPMPPTAGEAEQAADQQEREGALELLADLCENMDNAADFCQLSGMHLLVGRYLEAGAAGLRWRAAQLIGTCSQNVAAIQEQVLGLGALRKLLRLLDRDACDTVRVKALFAISCLVREQEAGLLQFLRLDGFSVLMRAMQQQVQKLKVKSAFLLQNLLVGHPEHKGTLCSMGMVQQLVALVRTEHSPFHEHVLGALCSLVTDFPQGVRECREPELGLEELLRHRCQLLQQHEEYQEELEFCEKLLQTCFSSPADDSMDR",
# CH60_HUMAN
"MLRLPTVFRQMRPVSRVLAPHLTRAYAKDVKFGADARALMLQGVDLLADAVAVTMGPKGRTVIIEQSWGSPKVTKDGVTVAKSIDLKDKYKNIGAKLVQDVANNTNEEAGDGTTTATVLARSIAKEGFEKISKGANPVEIRRGVMLAVDAVIAELKKQSKPVTTPEEIAQVATISANGDKEIGNIISDAMKKVGRKGVITVKDGKTLNDELEIIEGMKFDRGYISPYFINTSKGQKCEFQDAYVLLSEKKISSIQSIVPALEIANAHRKPLVIIAEDVDGEALSTLVLNRLKVGLQVVAVKAPGFGDNRKNQLKDMAIATGGAVFGEEGLTLNLEDVQPHDLGKVGEVIVTKDDAMLLKGKGDKAQIEKRIQEIIEQLDVTTSEYEKEKLNERLAKLSDGVAVLKVGGTSDVEVNEKKDRVTDALNATRAAVEEGIVLGGGCALLRCIPALDSLTPANEDQKIGIEIIKRTLKIPAMTIAKNAGVEGSLIVEKIMQSSSEVGYDAMAGDFVNMVEKGIIDPTKVVRTALLDAAGVASLLTTAEVVVTEIPKEEKDPGMGAMGGMGGGMGGGMF",
# CH10_HUMAN
"MAGQAFRKFLPLFDRVLVERSAAETVTKGGIMLPEKSQGKVLQATVVAVGSGSKGKGGEIQPVSVKVGDKVLLPEYGGTKVVLDDKDYFLFRDGDILGKYVD",
# HS105_HUMAN
"MSVVGLDVGSQSCYIAVARAGGIETIANEFSDRCTPSVISFGSKNRTIGVAAKNQQITHANNTVSNFKRFHGRAFNDPFIQKEKENLSYDLVPLKNGGVGIKVMYMGEEHLFSVEQITAMLLTKLKETAENSLKKPVTDCVISVPSFFTDAERRSVLDAAQIVGLNCLRLMNDMTAVALNYGIYKQDLPSLDEKPRIVVFVDMGHSAFQVSACAFNKGKLKVLGTAFDPFLGGKNFDEKLVEHFCAEFKTKYKLDAKSKIRALLRLYQECEKLKKLMSSNSTDLPLNIECFMNDKDVSGKMNRSQFEELCAELLQKIEVPLYSLLEQTHLKVEDVSAVEIVGGATRIPAVKERIAKFFGKDISTTLNADEAVARGCALQCAILSPAFKVREFSVTDAVPFPISLIWNHDSEDTEGVHEVFSRNHAAPFSKVLTFLRRGPFELEAFYSDPQGVPYPEAKIGRFVVQNVSAQKDGEKSRVKVKVRVNTHGIFTISTASMVEKVPTEENEMSSEADMECLNQRPPENPDTDKNVQQDNSEAGTQPQVQTDAQQTSQSPPSPELTSEENKIPDADKANEKKVDQPPEAKKPKIKVVNVELPIEANLVWQLGKDLLNMYIETEGKMIMQDKLEKERNDAKNAVEEYVYEFRDKLCGPYEKFICEQDHQNFLRLLTETEDWLYEEGEDQAKQAYVDKLEELMKIGTPVKVRFQEAEERPKMFEELGQRLQHYAKIAADFRNKDEKYNHIDESEMKKVEKSVNEVMEWMNNVMNAQAKKSLDQDPVVRAQEIKTKIKELNNTCEPVVTQPKPKIESPKLERTPNGPNIDKKEEDLEDKNNFGAEPPHQNGECYPNEKNSVNMDLD",
# HYOU1_HUMAN
"MADKVRRQRPRRRVCWALVAVLLADLLALSDTLAVMSVDLGSESMKVAIVKPGVPMEIVLNKESRRKTPVIVTLKENERFFGDSAASMAIKNPKATLRYFQHLLGKQADNPHVALYQARFPEHELTFDPQRQTVHFQISSQLQFSPEEVLGMVLNYSRSLAEDFAEQPIKDAVITVPVFFNQAERRAVLQAARMAGLKVLQLINDNTATALSYGVFRRKDINTTAQNIMFYDMGSGSTVCTIVTYQMVKTKEAGMQPQLQIRGVGFDRTLGGLEMELRLRERLAGLFNEQRKGQRAKDVRENPRAMAKLLREANRLKTVLSANADHMAQIEGLMDDVDFKAKVTRVEFEELCADLFERVPGPVQQALQSAEMSLDEIEQVILVGGATRVPRVQEVLLKAVGKEELGKNINADEAAAMGAVYQAAALSKAFKVKPFVVRDAVVYPILVEFTREVEEEPGIHSLKHNKRVLFSRMGPYPQRKVITFNRYSHDFNFHINYGDLGFLGPEDLRVFGSQNLTTVKLKGVGDSFKKYPDYESKGIKAHFNLDESGVLSLDRVESVFETLVEDSAEEESTLTKLGNTISSLFGGGTTPDAKENGTDTVQEEEESPAEGSKDEPGEQVELKEEAEAPVEDGSQPPPPEPKGDATPEGEKATEKENGDKSEAQKPSEKAEAGPEGVAPAPEGEKKQKPARKRRMVEEIGVELVVLDLPDLPEDKLAQSVQKLQDLTLRDLEKQEREKAANSLEAFIFETQDKLYQPEYQEVSTEEQREEISGKLSAASTWLEDEGVGATTVMLKEKLAELRKLCQGLFFRVEERKKWPERLSALDNLLNHSSMFLKGARLIPEMDQIFTEVEMTTLEKVINETWAWKNATLAEQAKLPATEKPVLLSKDIEAKMMALDREVQYLLNKAKFTKPRPRPKDKNGTRAEPPLNASASDQGEKVIPPAGQTEDAEPISEPEKVETGSEPGDTEPLELGGPGAEPEQKEQSTGQKRPLKNDEL",
# MKKS_HUMAN
"MSRLEAKKPSLCKSEPLTTERVRTTLSVLKRIVTSCYGPSGRLKQLHNGFGGYVCTTSQSSALLSHLLVTHPILKILTASIQNHVSSFSDCGLFTAILCCNLIENVQRLGLTPTTVIRLNKHLLSLCISYLKSETCGCRIPVDFSSTQILLCLVRSILTSKPACMLTRKETEHVSALILRAFLLTIPENAEGHIILGKSLIVPLKGQRVIDSTVLPGILIEMSEVQLMRLLPIKKSTALKVALFCTTLSGDTSDTGEGTVVVSYGVSLENAVLDQLLNLGRQLISDHVDLVLCQKVIHPSLKQFLNMHRIIAIDRIGVTLMEPLTKMTGTQPIGSLGSICPNSYGSVKDVCTAKFGSKHFFHLIPNEATICSLLLCNRNDTAWDELKLTCQTALHVLQLTLKEPWALLGGGCTETHLAAYIRHKTHNDPESILKDDECTQTELQLIAEAFCSALESVVGSLEHDGGEILTDMKYGHLWSVQADSPCVANWPDLLSQCGCGLYNSQEELNWSFLRSTRRPFVPQSCLPHEAVGSASNLTLDCLTAKLSGLQVAVETANLILDLSYVIEDKN",
# MZB1_HUMAN
"MRLSLPLLLLLLGAWAIPGGLGDRAPLTATAPQLDDEEMYSAHMPAHLRCDACRAVAYQMWQNLAKAETKLHTSNSGGRRELSELVYTDVLDRSCSRNWQDYGVREVDQVKRLTGPGLSEGPEPSISVMVTGGPWPTRLSRTCLHYLGEFGEDQIYEAHQQGRGALEALLCGGPQGACSEKVSATREEL",
# NUDC_HUMAN
"MGGEQEEERFDGMLLAMAQQHEGGVQELVNTFFSFLRRKTDFFIGGEEGMAEKLITQTFSHHNQLAQKTRREKRARQEAERREKAERAARLAKEAKSETSGPQIKELTDEEAERLQLEIDQKKDAENHEAQLKNGSLDSPGKQDTEEDEEEDEKDKGKLKPNLGNGADLPNYRWTQTLSELDLAVPFCVNFRLKGKDMVVDIQRRHLRVGLKGQPAIIDGELYNEVKVEESSWLIEDGKVVTVHLEKINKMEWWSRLVSSDPEINTKKINPENSKLSDLDSETRSMVEKMMYDQRQKSMGLPTSDEQKKQEILKKFMDQHPEMDFSKAKFN",
# ODFP1_HUMAN
"MAALSCLLDSVRRDIKKVDRELRQLRCIDEFSTRCLCDLYMHPYCCCDLHPYPYCLCYSKRSRSCGLCDLYPCCLCDYKLYCLRPSLRSLERKAIRAIEDEKRELAKLRRTTNRILASSCCSSNILGSVNVCGFEPDQVKVRVKDGKVCVSAERENRYDCLGSKKYSYMNICKEFSLPPCVDEKDVTYSYGLGSCVKIESPCYPCTSPCSPCSPCSPCNPCSPCNPCSPYDPCNPCYPCGSRFSCRKMIL",
# PDIA1_HUMAN
"MLRRALLCLAVAALVRADAPEEEDHVLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLKAEGSEIRLAKVDATEESDLAQQYGVRGYPTIKFFRNGDTASPKEYTAGREADDIVNWLKKRTGPAATTLPDGAAAESLVESSEVAVIGFFKDVESDSAKQFLQAAEAIDDIPFGITSNSDVFSKYQLDKDGVVLFKKFDEGRNNFEGEVTKENLLDFIKHNQLPLVIEFTEQTAPKIFGGEIKTHILLFLPKSVSDYDGKLSNFKTAAESFKGKILFIFIDSDHTDNQRILEFFGLKKEECPAVRLITLEEEMTKYKPESEELTAERITEFCHRFLEGKIKPHLMSQELPEDWDKQPVKVLVGKNFEDVAFDEKKNVFVEFYAPWCGHCKQLAPIWDKLGETYKDHENIVIAKMDSTANEVEAVKVHSFPTLKFFPASADRTVIDYNGERTLDGFKKFLESGGQDGAGDDDDLEDLEEAEEPDMEEDDDQKAVKDEL",
# PDIA2_HUMAN
"MSRQLLPVLLLLLLRASCPWGQEQGARSPSEEPPEEEIPKEDGILVLSRHTLGLALREHPALLVEFYAPWCGHCQALAPEYSKAAAVLAAESMVVTLAKVDGPAQRELAEEFGVTEYPTLKFFRNGNRTHPEEYTGPRDAEGIAEWLRRRVGPSAMRLEDEAAAQALIGGRDLVVIGFFQDLQDEDVATFLALAQDALDMTFGLTDRPRLFQQFGLTKDTVVLFKKFDEGRADFPVDEELGLDLGDLSRFLVTHSMRLVTEFNSQTSAKIFAARILNHLLLFVNQTLAAHRELLAGFGEAAPRFRGQVLFVVVDVAADNEHVLQYFGLKAEAAPTLRLVNLETTKKYAPVDGGPVTAASITAFCHAVLNGQVKPYLLSQEIPPDWDQRPVKTLVGKNFEQVAFDETKNVFVKFYAPWCTHCKEMAPAWEALAEKYQDHEDIIIAELDATANELDAFAVHGFPTLKYFPAGPGRKVIEYKSTRDLETFSKFLDNGGVLPTEEPPEEPAAPFPEPPANSTMGSKEEL",
# PDIA3_HUMAN
"MRLRRLALFPGVALLLAAARLAAASDVLELTDDNFESRISDTGSAGLMLVEFFAPWCGHCKRLAPEYEAAATRLKGIVPLAKVDCTANTNTCNKYGVSGYPTLKIFRDGEEAGAYDGPRTADGIVSHLKKQAGPASVPLRTEEEFKKFISDKDASIVGFFDDSFSEAHSEFLKAASNLRDNYRFAHTNVESLVNEYDDNGEGIILFRPSHLTNKFEDKTVAYTEQKMTSGKIKKFIQENIFGICPHMTEDNKDLIQGKDLLIAYYDVDYEKNAKGSNYWRNRVMMVAKKFLDAGHKLNFAVASRKTFSHELSDFGLESTAGEIPVVAIRTAKGEKFVMQEEFSRDGKALERFLQDYFDGNLKRYLKSEPIPESNDGPVKVVVAENFDEIVNNENKDVLIEFYAPWCGHCKNLEPKYKELGEKLSKDPNIVIAKMDATANDVPSPYEVRGFPTIYFSPANKKLNPKKYEGGRELSDFISYLQREATNPPVIQEEKPKKKKKAQEDL",
# PDIA4_HUMAN
"MRPRKAFLLLLLLGLVQLLAVAGAEGPDEDSSNRENAIEDEEEEEEEDDDEEEDDLEVKEENGVLVLNDANFDNFVADKDTVLLEFYAPWCGHCKQFAPEYEKIANILKDKDPPIPVAKIDATSASVLASRFDVSGYPTIKILKKGQAVDYEGSRTQEEIVAKVREVSQPDWTPPPEVTLVLTKENFDEVVNDADIILVEFYAPWCGHCKKLAPEYEKAAKELSKRSPPIPLAKVDATAETDLAKRFDVSGYPTLKIFRKGRPYDYNGPREKYGIVDYMIEQSGPPSKEILTLKQVQEFLKDGDDVIIIGVFKGESDPAYQQYQDAANNLREDYKFHHTFSTEIAKFLKVSQGQLVVMQPEKFQSKYEPRSHMMDVQGSTQDSAIKDFVLKYALPLVGHRKVSNDAKRYTRRPLVVVYYSVDFSFDYRAATQFWRSKVLEVAKDFPEYTFAIADEEDYAGEVKDLGLSESGEDVNAAILDESGKKFAMEPEEFDSDTLREFVTAFKKGKLKPVIKSQPVPKNNKGPVKVVVGKTFDSIVMDPKKDVLIEFYAPWCGHCKQLEPVYNSLAKKYKGQKGLVIAKMDATANDVPSDRYKVEGFPTIYFAPSGDKKNPVKFEGGDRDLEHLSKFIEEHATKLSRTKEEL",
# PDIA5_HUMAN
"MARAGPAWLLLAIWVVLPSWLSSAKVSSLIERISDPKDLKKLLRTRNNVLVLYSKSEVAAENHLRLLSTVAQAVKGQGTICWVDCGDAESRKLCKKMKVDLSPKDKKVELFHYQDGAFHTEYNRAVTFKSIVAFLKDPKGPPLWEEDPGAKDVVHLDSEKDFRRLLKKEEKPLLIMFYAPWCSMCKRMMPHFQKAATQLRGHAVLAGMNVYSSEFENIKEEYSVRGFPTICYFEKGRFLFQYDNYGSTAEDIVEWLKNPQPPQPQVPETPWADEGGSVYHLTDEDFDQFVKEHSSVLVMFHAPWCGHCKKMKPEFEKAAEALHGEADSSGVLAAVDATVNKALAERFHISEFPTLKYFKNGEKYAVPVLRTKKKFLEWMQNPEAPPPPEPTWEEQQTSVLHLVGDNFRETLKKKKHTLVMFYAPWCPHCKKVIPHFTATADAFKDDRKIACAAVDCVKDKNQDLCQQEAVKGYPTFHYYHYGKFAEKYDSDRTELGFTNYIRALREGDHERLGKKKEEL",
# PDIA6_HUMAN
"MALLVLGLVSCTFFLAVNGLYSSSDDVIELTPSNFNREVIQSDSLWLVEFYAPWCGHCQRLTPEWKKAATALKDVVKVGAVDADKHHSLGGQYGVQGFPTIKIFGSNKNRPEDYQGGRTGEAIVDAALSALRQLVKDRLGGRSGGYSSGKQGRSDSSSKKDVIELTDDSFDKNVLDSEDVWMVEFYAPWCGHCKNLEPEWAAAASEVKEQTKGKVKLAAVDATVNQVLASRYGIRGFPTIKIFQKGESPVDYDGGRTRSDIVSRALDLFSDNAPPPELLEIINEDIAKRTCEEHQLCVVAVLPHILDTGAAGRNSYLEVLLKLADKYKKKMWGWLWTEAGAQSELETALGIGGFGYPAMAAINARKMKFALLKGSFSEQGINEFLRELSFGRGSTAPVGGGAFPTIVEREPWDGRDGELPVEDDIDLSDVELDDLGKDEL",
# PDRG1_HUMAN
"MLSPEAERVLRYLVEVEELAEEVLADKRQIVDLDTKRNQNREGLRALQKDLSLSEDVMVCFGNMFIKMPHPETKEMIEKDQDHLDKEIEKLRKQLKVKVNRLFEAQGKPELKGFNLNPLNQDELKALKVILKG",
# PFD1_HUMAN
"MAAPVDLELKKAFTELQAKVIDTQQKVKLADIQIEQLNRTKKHAHLTDTEIMTLVDETNMYEGVGRMFILQSKEAIHSQLLEKQKIAEEKIKELEQKKSYLERSVKEAEDNIREMLMARRAQ",
# PFD2_HUMAN
"MAENSGRAGKSSGSGAGKGAVSAEQVIAGFNRLRQEQRGLASKAAELEMELNEHSLVIDTLKEVDETRKCYRMVGGVLVERTVKEVLPALENNKEQIQKIIETLTQQLQAKGKELNEFREKHNIRLMGEDEKPAAKENSEGAGAKASSAGVLVS",
# PFD4_HUMAN
"MAATMKKAAAEDVNVTFEDQQKINKFARNTSRITELKEEIEVKKKQLQNLEDACDDIMLADDDCLMIPYQIGDVFISHSQEETQEMLEEAKKNLQEEIDALESRVESIQRVLADLKVQLYAKFGSNINLEADES",
# PFD5_HUMAN
"MAQSINITELNLPQLEMLKNQLDQEVEFLSTSIAQLKVVQTKYVEAKDCLNVLNKSNEGKELLVPLTSSMYVPGKLHDVEHVLIDVGTGYYVEKTAEDAKDFFKRKIDFLTKQMEKIQPALQEKHAMKQAVMEMMSQKIQQLTALGAAQATAKA",
# PFD6_HUMAN
"MAELIQKKLQGEVEKYQQLQKDLSKSMSGRQKLEAQLTENNIVKEELALLDGSNVVFKLLGPVLVKQELGEARATVGKRLDYITAEIKRYESQLRDLERQSEQQRETLAQLQQEFQRAQAAKAGAPGKA",
# PHB1_HUMAN
"MAAKVFESIGKFGLALAVAGGVVNSALYNVDAGHRAVIFDRFRGVQDIVVGEGTHFLIPWVQKPIIFDCRSRPRNVPVITGSKDLQNVNITLRILFRPVASQLPRIFTSIGEDYDERVLPSITTEILKSVVARFDAGELITQRELVSRQVSDDLTERAATFGLILDDVSLTHLTFGKEFTEAVEAKQVAQQEAERARFVVEKAEQQKKAAIISAEGDSKAAELIANSLATAGDGLIELRKLEAAEDIAYQLSRSRNITYLPAGQSVLLQLPQ",
# PHB2_HUMAN
"MAQNLKDLAGRLPAGPRGMGTALKLLLGAGAVAYGVRESVFTVEGGHRAIFFNRIGGVQQDTILAEGLHFRIPWFQYPIIYDIRARPRKISSPTGSKDLQMVNISLRVLSRPNAQELPSMYQRLGLDYEERVLPSIVNEVLKSVVAKFNASQLITQRAQVSLLIRRELTERAKDFSLILDDVAITELSFSREYTAAVEAKQVAQQEAQRAQFLVEKAKQEQRQKIVQAEGEAEAAKMLGEALSKNPGYIKLRKIRAAQNISKTIATSQNRIYLTADNLVLNLQDESFTRGSDSLIKGKK",
# PIHD1_HUMAN
"MANPKLLGMGLSEAEAIGADSARFEELLLQASKELQQAQTTRPESTQIQPQPGFCIKTNSSEGKVFINICHSPSIPPPADVTEEELLQMLEEDQAGFRIPMSLGEPHAELDAKGQGCTAYDVAVNSDFYRRMQNSDFLRELVITIAREGLEDKYNLQLNPEWRMMKNRPFMGSISQQNIRSEQRPRIQELGDLYTPAPGRAESGPEKPHLNLWLEAPDLLLAEVDLPKLDGALGLSLEIGENRLVMGGPQQLYHLDAYIPLQINSHESKAAFHRKRKQLMVAMPLLPVPS",
# PPIA_HUMAN
"MVNPTVFFDIAVDGEPLGRVSFELFADKVPKTAENFRALSTGEKGFGYKGSCFHRIIPGFMCQGGDFTRHNGTGGKSIYGEKFEDENFILKHTGPGILSMANAGPNTNGSQFFICTAKTEWLDGKHVVFGKVKEGMNIVEAMERFGSRNGKTSKKITIADCGQLE",
# PAL4A_HUMAN
"MVNSVVFFDITVDGKPLGRISIKLFADKILKTAENFRALSTGEKGFRYKGSCFHRIIPGFMCQGGDFTRHNGTGDKSIYGEKFDDENLIRKHTGSGILSMANAGPNTNGSQFFICAAKTEWLDGKHVAFGKVKERVNIVEAMEHFGYRNSKTSKKITIADCGQF",
# PPIB_HUMAN
"MLRLSERNMKVLLAAALIAGSVFFLLLPGPSAADEKKKGPKVTVKVYFDLRIGDEDVGRVIFGLFGKTVPKTVDNFVALATGEKGFGYKNSKFHRVIKDFMIQGGDFTRGDGTGGKSIYGERFPDENFKLKHYGPGWVSMANAGKDTNGSQFFITTVKTAWLDGKHVVFGKVLEGMEVVRKVESTKTDSRDKPLKDVIIADCGKIEVEKPFAIAKE",
# PPIC_HUMAN
"MGPGPRLLLPLVLCVGLGALVFSSGAEGFRKRGPSVTAKVFFDVRIGDKDVGRIVIGLFGKVVPKTVENFVALATGEKGYGYKGSKFHRVIKDFMIQGGDITTGDGTGGVSIYGETFPDENFKLKHYGIGWVSMANAGPDTNGSQFFITLTKPTWLDGKHVVFGKVIDGMTVVHSIELQATDGHDRPLTNCSIINSGKIDVKTPFVVEIADW",
# PPID_HUMAN
"MSHPSPQAKPSNPSNPRVFFDVDIGGERVGRIVLELFADIVPKTAENFRALCTGEKGIGHTTGKPLHFKGCPFHRIIKKFMIQGGDFSNQNGTGGESIYGEKFEDENFHYKHDREGLLSMANAGRNTNGSQFFITTVPTPHLDGKHVVFGQVIKGIGVARILENVEVKGEKPAKLCVIAECGELKEGDDGGIFPKDGSGDSHPDFPEDADIDLKDVDKILLITEDLKNIGNTFFKSQNWEMAIKKYAEVLRYVDSSKAVIETADRAKLQPIALSCVLNIGACKLKMSNWQGAIDSCLEALELDPSNTKALYRRAQGWQGLKEYDQALADLKKAQGIAPEDKAIQAELLKVKQKIKAQKDKEKAVYAKMFA",
# PPP5_HUMAN
"MAMAEGERTECAEPPRDEPPADGALKRAEELKTQANDYFKAKDYENAIKFYSQAIELNPSNAIYYGNRSLAYLRTECYGYALGDATRAIELDKKYIKGYYRRAASNMALGKFRAALRDYETVVKVKPHDKDAKMKYQECNKIVKQKAFERAIAGDEHKRSVVDSLDIESMTIEDEYSGPKLEDGKVTISFMKELMQWYKDQKKLHRKCAYQILVQVKEVLSKLSTLVETTLKETEKITVCGDTHGQFYDLLNIFELNGLPSETNPYIFNGDFVDRGSFSVEVILTLFGFKLLYPDHFHLLRGNHETDNMNQIYGFEGEVKAKYTAQMYELFSEVFEWLPLAQCINGKVLIMHGGLFSEDGVTLDDIRKIERNRQPPDSGPMCDLLWSDPQPQNGRSISKRGVSCQFGPDVTKAFLEENNLDYIIRSHEVKAEGYEVAHGGRCVTVFSAPNYCDQMGNKASYIHLQGSDLRPQFHQFTAVPHPNVKPMAYANTLLQLGMM",
# TEBP_HUMAN
"MQPASAKWYDRRDYVFIEFCVEDSKDVNVNFEKSKLTFSCLGGSDNFKHLNEIDLFHCIDPNDSKHKRTDRSILCCLRKGESGQSWPRLTKERAKLNWLSVDFNNWKDWEDDSDEDMSNFDRFSEMMNNMGGDEDVDLPEVDGADDDSQDSDDEKMPDLE",
# RPAP3_HUMAN
"MTSANKAIELQLQVKQNAEELQDFMRDLENWEKDIKQKDMELRRQNGVPEENLPPIRNGNFRKKKKGKAKESSKKTREENTKNRIKSYDYEAWAKLDVDRILDELDKDDSTHESLSQESESEEDGIHVDSQKALVLKEKGNKYFKQGKYDEAIDCYTKGMDADPYNPVLPTNRASAYFRLKKFAVAESDCNLAVALNRSYTKAYSRRGAARFALQKLEEAKKDYERVLELEPNNFEATNELRKISQALASKENSYPKEADIVIKSTEGERKQIEAQQNKQQAISEKDRGNGFFKEGKYERAIECYTRGIAADGANALLPANRAMAYLKIQKYEEAEKDCTQAILLDGSYSKAFARRGTARTFLGKLNEAKQDFETVLLLEPGNKQAVTELSKIKKELIEKGHWDDVFLDSTQRQNVVKPIDNPPHPGSTKPLKKVIIEETGNLIQTIDVPDSTTAAAPENNPINLANVIAATGTTSKKNSSQDDLFPTSDTPRAKVLKIEEVSDTSSLQPQASLKQDVCQSYSEKMPIEIEQKPAQFATTVLPPIPANSFQLESDFRQLKSSPDMLYQYLKQIEPSLYPKLFQKNLDPDVFNQIVKILHDFYIEKEKPLLIFEILQRLSELKRFDMAVMFMSETEKKIARALFNHIDKSGLKDSSVEELKKRYGG",
# RUVB1_HUMAN
"MKIEEVKSTTKTQRIASHSHVKGLGLDESGLAKQAASGLVGQENAREACGVIVELIKSKKMAGRAVLLAGPPGTGKTALALAIAQELGSKVPFCPMVGSEVYSTEIKKTEVLMENFRRAIGLRIKETKEVYEGEVTELTPCETENPMGGYGKTISHVIIGLKTAKGTKQLKLDPSIFESLQKERVEAGDVIYIEANSGAVKRQGRCDTYATEFDLEAEEYVPLPKGDVHKKKEIIQDVTLHDLDVANARPQGGQDILSMMGQLMKPKKTEITDKLRGEINKVVNKYIDQGIAELVPGVLFVDEVHMLDIECFTYLHRALESSIAPIVIFASNRGNCVIRGTEDITSPHGIPLDLLDRVMIIRTMLYTPQEMKQIIKIRAQTEGINISEEALNHLGEIGTKTTLRYSVQLLTPANLLAKINGKDSIEKEHVEEISELFYDAKSSAKILADQQDKYMK",
# RUVB2_HUMAN
"MATVTATTKVPEIRDVTRIERIGAHSHIRGLGLDDALEPRQASQGMVGQLAARRAAGVVLEMIREGKIAGRAVLIAGQPGTGKTAIAMGMAQALGPDTPFTAIAGSEIFSLEMSKTEALTQAFRRSIGVRIKEETEIIEGEVVEIQIDRPATGTGSKVGKLTLKTTEMETIYDLGTKMIESLTKDKVQAGDVITIDKATGKISKLGRSFTRARDYDAMGSQTKFVQCPDGELQKRKEVVHTVSLHEIDVINSRTQGFLALFSGDTGEIKSEVREQINAKVAEWREEGKAEIIPGVLFIDEVHMLDIESFSFLNRALESDMAPVLIMATNRGITRIRGTSYQSPHGIPIDLLDRLLIVSTTPYSEKDTKQILRIRCEEEDVEMSEDAYTVLTRIGLETSLRYAIQLITAASLVCRKRKGTEVQVDDIKRVYSLFLDESRSTQYMKEYQDAFLFNELKGETMDTS",
# SACS_HUMAN
"METKENRWVPVTVLPGCVGCRTVAALASWTVRDVKERIFAETGFPVSEQRLWRGGRELSDWIKIGDLTSKNCHLFVNLQSKGLKGGGRFGQTTPPLVDFLKDILRRYPEGGQILKELIQNAEDAGATEVKFLYDETQYGTETLWSKDMAPYQGPALYVYNNAVFTPEDWHGIQEIARSRKKDDPLKVGRFGIGFNSVYHITDVPCIFSGDQIGMLDPHQTLFGPHESGQCWNLKDDSKEISELSDQFAPFVGIFGSTKETFINGNFPGTFFRFPLRLQPSQLSSNLYNKQKVLELFESFRADADTVLLFLKSVQDVSLYVREADGTEKLVFRVTSSESKALKHERPNSIKILGTAISNYCKKTPSNNITCVTYHVNIVLEEESTKDAQKTSWLVCNSVGGRGISSKLDSLADELKFVPIIGIAMPLSSRDDEAKGATSDFSGKAFCFLPLPPGEESSTGLPVHISGFFGLTDNRRSIKWRELDQWRDPAALWNEFLVMNVVPKAYATLILDSIKRLEMEKSSDFPLSVDVIYKLWPEASKVKVHWQPVLEPLFSELLQNAVIYSISCDWVRLEQVYFSELDENLEYTKTVLNYLQSSGKQIAKVPGNVDAAVQLTAASGTTPVRKVTPAWVRQVLRKCAHLGCAEEKLHLLEFVLSDQAYSELLGLELLPLQNGNFVPFSSSVSDQDVIYITSAEYPRSLFPSLEGRFILDNLKPHLVAALKEAAQTRGRPCTQLQLLNPERFARLIKEVMNTFWPGRELIVQWYPFDENRNHPSVSWLKMVWKNLYIHFSEDLTLFDEMPLIPRTILEEGQTCVELIRLRIPSLVILDDESEAQLPEFLADIVQKLGGFVLKKLDASIQHPLIKKYIHSPLPSAVLQIMEKMPLQKLCNQITSLLPTHKDALRKFLASLTDSSEKEKRIIQELAIFKRINHSSDQGISSYTKLKGCKVLHHTAKLPADLRLSISVIDSSDEATIRLANMLKIEQLKTTSCLKLVLKDIENAFYSHEEVTQLMLWVLENLSSLKNENPNVLEWLTPLKFIQISQEQMVSAGELFDPDIEVLKDLFCNEEGTYFPPSVFTSPDILHSLRQIGLKNEASLKEKDVVQVAKKIEALQVGACPDQDVLLKKAKTLLLVLNKNHTLLQSSEGKMTLKKIKWVPACKERPPNYPGSLVWKGDLCNLCAPPDMCDVGHAILIGSSLPLVESIHVNLEKALGIFTKPSLSAVLKHFKIVVDWYSSKTFSDEDYYQFQHILLEIYGFMHDHLNEGKDSFRALKFPWVWTGKKFCPLAQAVIKPIHDLDLQPYLHNVPKTMAKFHQLFKVCGSIEELTSDHISMVIQKIYLKSDQDLSEQESKQNLHLMLNIIRWLYSNQIPASPNTPVPIHHSKNPSKLIMKPIHECCYCDIKVDDLNDLLEDSVEPIILVHEDIPMKTAEWLKVPCLSTRLINPENMGFEQSGQREPLTVRIKNILEEYPSVSDIFKELLQNADDANATECSFLIDMRRNMDIRENLLDPGMAACHGPALWSFNNSQFSDSDFVNITRLGESLKRGEVDKVGKFGLGFNSVYHITDIPIIMSREFMIMFDPNINHISKHIKDKSNPGIKINWSKQQKRLRKFPNQFKPFIDVFGCQLPLTVEAPYSYNGTLFRLSFRTQQEAKVSEVSSTCYNTADIYSLVDEFSLCGHRLIIFTQSVKSMYLKYLKIEETNPSLAQDTVIIKKKSCSSKALNTPVLSVLKEAAKLMKTCSSSNKKLPSDEPKSSCILQITVEEFHHVFRRIADLQSPLFRGPDDDPAALFEMAKSGQSKKPSDELSQKTVECTTWLLCTCMDTGEALKFSLSESGRRLGLVPCGAVGVQLSEIQDQKWTVKPHIGEVFCYLPLRIKTGLPVHINGCFAVTSNRKEIWKTDTKGRWNTTFMRHVIVKAYLQVLSVLRDLATSGELMDYTYYAVWPDPDLVHDDFSVICQGFYEDIAHGKGKELTKVFSDGSTWVSMKNVRFLDDSILKRRDVGSAAFKIFLKYLKKTGSKNLCAVELPSSVKLGFEEAGCKQILLENTFSEKQFFSEVFFPNIQEIEAELRDPLMIFVLNEKVDEFSGVLRVTPCIPCSLEGHPLVLPSRLIHPEGRVAKLFDIKDGRFPYGSTQDYLNPIILIKLVQLGMAKDDILWDDMLERAVSVAEINKSDHVAACLRSSILLSLIDEKLKIRDPRAKDFAAKYQTIRFLPFLTKPAGFSLDWKGNSFKPETMFAATDLYTAEHQDIVCLLQPILNENSHSFRGCGSVSLAVKEFLGLLKKPTVDLVINQLKEVAKSVDDGITLYQENITNACYKYLHEALMQNEITKMSIIDKLKPFSFILVENAYVDSEKVSFHLNFEAAPYLYQLPNKYKNNFRELFETVGVRQSCTVEDFALVLESIDQERGTKQITEENFQLCRRIISEGIWSLIREKKQEFCEKNYGKILLPDTNLMLLPAKSLCYNDCPWIKVKDTTVKYCHADIPREVAVKLGAVPKRHKALERYASNVCFTTLGTEFGQKEKLTSRIKSILNAYPSEKEMLKELLQNADDAKATEICFVFDPRQHPVDRIFDDKWAPLQGPALCVYNNQPFTEDDVRGIQNLGKGTKEGNPYKTGQYGIGFNSVYHITDCPSFISGNDILCIFDPHARYAPGATSISPGRMFRDLDADFRTQFSDVLDLYLGTHFKLDNCTMFRFPLRNAEMAKVSEISSVPASDRMVQNLLDKLRSDGAELLMFLNHMEKISICEIDKSTGALNVLYSVKGKITDGDRLKRKQFHASVIDSVTKKRQLKDIPVQQITYTMDTEDSEGNLTTWLICNRSGFSSMEKVSKSVISAHKNQDITLFPRGGVAACITHNYKKPHRAFCFLPLSLETGLPFHVNGHFALDSARRNLWRDDNGVGVRSDWNNSLMTALIAPAYVELLIQLKKRYFPGSDPTLSVLQNTPIHVVKDTLKKFLSFFPVNRLDLQPDLYCLVKALYNCIHEDMKRLLPVVRAPNIDGSDLHSAVIITWINMSTSNKTRPFFDNLLQDELQHLKNADYNITTRKTVAENVYRLKHLLLEIGFNLVYNCDETANLYHCLIDADIPVSYVTPADIRSFLMTFSSPDTNCHIGKLPCRLQQTNLKLFHSLKLLVDYCFKDAEENEIEVEGLPLLITLDSVLQTFDAKRPKFLTTYHELIPSRKDLFMNTLYLKYSNILLNCKVAKVFDISSFADLLSSVLPREYKTKSCTKWKDNFASESWLKNAWHFISESVSVKEDQEETKPTFDIVVDTLKDWALLPGTKFTVSANQLVVPEGDVLLPLSLMHIAVFPNAQSDKVFHALMKAGCIQLALNKICSKDSAFVPLLSCHTANIESPTSILKALHYMVQTSTFRAEKLVENDFEALLMYFNCNLNHLMSQDDIKILKSLPCYKSISGRYVSIGKFGTCYVLTKSIPSAEVEKWTQSSSSAFLEEKIHLKELYEVIGCVPVDDLEVYLKHLLPKIENLSYDAKLEHLIYLKNRLSSAEELSEIKEQLFEKLESLLIIHDANSRLKQAKHFYDRTVRVFEVMLPEKLFIPNDFFKKLEQLIKPKNHVTFMTSWVEFLRNIGLKYILSQQQLLQFAKEISVRANTENWSKETLQNTVDILLHHIFQERMDLLSGNFLKELSLIPFLCPERAPAEFIRFHPQYQEVNGTLPLIKFNGAQVNPKFKQCDVLQLLWTSCPILPEKATPLSIKEQEGSDLGPQEQLEQVLNMLNVNLDPPLDKVINNCRNICNITTLDEEMVKTRAKVLRSIYEFLSAEKREFRFQLRGVAFVMVEDGWKLLKPEEVVINLEYESDFKPYLYKLPLELGTFHQLFKHLGTEDIISTKQYVEVLSRIFKNSEGKQLDPNEMRTVKRVVSGLFRSLQNDSVKVRSDLENVRDLALYLPSQDGRLVKSSILVFDDAPHYKSRIQGNIGVQMLVDLSQCYLGKDHGFHTKLIMLFPQKLRPRLLSSILEEQLDEETPKVCQFGALCSLQGRLQLLLSSEQFITGLIRIMKHENDNAFLANEEKAIRLCKALREGLKVSCFEKLQTTLRVKGFNPIPHSRSETFAFLKRFGNAVILLYIQHSDSKDINFLLALAMTLKSATDNLISDTSYLIAMLGCNDIYRIGEKLDSLGVKYDSSEPSKLELPMPGTPIPAEIHYTLLMDPMNVFYPGEYVGYLVDAEGGDIYGSYQPTYTYAIIVQEVEREDADNSSFLGKIYQIDIGYSEYKIVSSLDLYKFSRPEESSQSRDSAPSTPTSPTEFLTPGLRSIPPLFSGRESHKTSSKHQSPKKLKVNSLPEILKEVTSVVEQAWKLPESERKKIIRRLYLKWHPDKNPENHDIANEVFKHLQNEINRLEKQAFLDQNADRASRRTFSTSASRFQSDKYSFQRFYTSWNQEATSHKSERQQQNKEKCPPSAGQTYSQRFFVPPTFKSVGNPVEARRWLRQARANFSAARNDLHKNANEWVCFKCYLSTKLALIAADYAVRGKSDKDVKPTALAQKIEEYSQQLEGLTNDVHTLEAYGVDSLKTRYPDLLPFPQIPNDRFTSEVAMRVMECTACIIIKLENFMQQKV",
# SEC63_HUMAN
"MAGQQFQYDDSGNTFFYFLTSFVGLIVIPATYYLWPRDQNAEQIRLKNIRKVYGRCMWYRLRLLKPQPNIIPTVKKIVLLAGWALFLFLAYKVSKTDREYQEYNPYEVLNLDPGATVAEIKKQYRLLSLKYHPDKGGDEVMFMRIAKAYAALTDEESRKNWEEFGNPDGPQATSFGIALPAWIVDQKNSILVLLVYGLAFMVILPVVVGSWWYRSIRYSGDQILIRTTQIYTYFVYKTRNMDMKRLIMVLAGASEFDPQYNKDATSRPTDNILIPQLIREIGSINLKKNEPPLTCPYSLKARVLLLSHLARMKIPETLEEDQQFMLKKCPALLQEMVNVICQLIVMARNREEREFRAPTLASLENCMKLSQMAVQGLQQFKSPLLQLPHIEEDNLRRVSNHKKYKIKTIQDLVSLKESDRHTLLHFLEDEKYEEVMAVLGSFPYVTMDIKSQVLDDEDSNNITVGSLVTVLVKLTRQTMAEVFEKEQSICAAEEQPAEDGQGETNKNRTKGGWQQKSKGPKKTAKSKKKKPLKKKPTPVLLPQSKQQKQKQANGVVGNEAAVKEDEEEVSDKGSDSEEEETNRDSQSEKDDGSDRDSDREQDEKQNKDDEAEWQELQQSIQRKERALLETKSKITHPVYSLYFPEEKQEWWWLYIADRKEQTLISMPYHVCTLKDTEEVELKFPAPGKPGNYQYTVFLRSDSYMGLDQIKPLKLEVHEAKPVPENHPQWDTAIEGDEDQEDSEGFEDSFEEEEEEEEDDD",
# SERPH_HUMAN
"MRSLLLLSAFCLLEAALAAEVKKPAAAAAPGTAEKLSPKAATLAERSAGLAFSLYQAMAKDQAVENILVSPVVVASSLGLVSLGGKATTASQAKAVLSAEQLRDEEVHAGLGELLRSLSNSTARNVTWKLGSRLYGPSSVSFADDFVRSSKQHYNCEHSKINFRDKRSALQSINEWAAQTTDGKLPEVTKDVERTDGALLVNAMFFKPHWDEKFHHKMVDNRGFMVTRSYTVGVMMMHRTGLYNYYDDEKEKLQIVEMPLAHKLSSLIILMPHHVEPLERLEKLLTKEQLKIWMGKMQKKAVAISLPKGVVEVTHDLQKHLAGLGLTEAIDKNKADLSRMSGKKDLYLASVFHATAFELDTDGNPFDQDIYGREELRSPKLFYADHPFIFLVRDTQSGSLLFIGRLVRPKGDKMRDEL",
# SGTA_HUMAN
"MDNKKRLAYAIIQFLHDQLRHGGLSSDAQESLEVAIQCLETAFGVTVEDSDLALPQTLPEIFEAAATGKEMPQDLRSPARTPPSEEDSAEAERLKTEGNEQMKVENFEAAVHFYGKAIELNPANAVYFCNRAAAYSKLGNYAGAVQDCERAICIDPAYSKAYGRMGLALSSLNKHVEAVAYYKKALELDPDNETYKSNLKIAELKLREAPSPTGGVGSFDIAGLLNNPGFMSMASNLMNNPQIQQLMSGMISGGNNPLGTPGTSPSQNDLASLIQAGQQFAQQMQQQNPELIEQLRSQIRSRTPSASNDDQQE",
# SGTB_HUMAN
"MSSIKHLVYAVIRFLREQSQMDTYTSDEQESLEVAIQCLETVFKISPEDTHLAVSQPLTEMFTSSFCKNDVLPLSNSVPEDVGKADQLKDEGNNHMKEENYAAAVDCYTQAIELDPNNAVYYCNRAAAQSKLGHYTDAIKDCEKAIAIDSKYSKAYGRMGLALTALNKFEEAVTSYQKALDLDPENDSYKSNLKIAEQKLREVSSPTGTGLSFDMASLINNPAFISMAASLMQNPQVQQLMSGMMTNAIGGPAAGVGGLTDLSSLIQAGQQFAQQIQQQNPELIEQLRNHIRSRSFSSSAEEHS",
# SIL1_HUMAN
"MAPQSLPSSRMAPLGMLLGLLMAACFTFCLSHQNLKEFALTNPEKSSTKETERKETKAEEELDAEVLEVFHPTHEWQALQPGQAVPAGSHVRLNLQTGEREAKLQYEDKFRNNLKGKRLDINTNTYTSQDLKSALAKFKEGAEMESSKEDKARQAEVKRLFRPIEELKKDFDELNVVIETDMQIMVRLINKFNSSSSSLEEKIAALFDLEYYVHQMDNAQDLLSFGGLQVVINGLNSTEPLVKEYAAFVLGAAFSSNPKVQVEAIEGGALQKLLVILATEQPLTAKKKVLFALCSLLRHFPYAQRQFLKLGGLQVLRTLVQEKGTEVLAVRVVTLLYDLVTEKMFAEEEAELTQEMSPEKLQQYRQVHLLPGLWEQGWCEITAHLLALPEHDAREKVLQTLGVLLTTCRDRYRQDPQLGRTLASLQAEYQVLASLELQDGEDEGYFQELLGSVNSLLKELR",
# F10A1_HUMAN
"MDPRKVNELRAFVKMCKQDPSVLHTEEMRFLREWVESMGGKVPPATQKAKSEENTKEEKPDSKKVEEDLKADEPSSEESDLEIDKEGVIEPDTDAPQEMGDENAEITEEMMDQANDKKVAAIEALNDGELQKAIDLFTDAIKLNPRLAILYAKRASVFVKLQKPNAAIRDCDRAIEINPDSAQPYKWRGKAHRLLGHWEEAAHDLALACKLDYDEDASAMLKEVQPRAQKIAEHRRKYERKREEREIKERIERVKKAREEHERAQREEEARRQSGAQYGSFPGGFPGGMPGNFPGGMPGMGGGMPGMAGMPGLNEILSDPEVLAAMQDPEVMVAFQDVAQNPANMSKYQSNPKVMNLISKLSAKFGGQA",
# STIP1_HUMAN
"MEQVNELKEKGNKALSVGNIDDALQCYSEAIKLDPHNHVLYSNRSAAYAKKGDYQKAYEDGCKTVDLKPDWGKGYSRKAAALEFLNRFEEAKRTYEEGLKHEANNPQLKEGLQNMEARLAERKFMNPFNMPNLYQKLESDPRTRTLLSDPTYRELIEQLRNKPSDLGTKLQDPRIMTTLSVLLGVDLGSMDEEEEIATPPPPPPPKKETKPEPMEEDLPENKKQALKEKELGNDAYKKKDFDTALKHYDKAKELDPTNMTYITNQAAVYFEKGDYNKCRELCEKAIEVGRENREDYRQIAKAYARIGNSYFKEEKYKDAIHFYNKSLAEHRTPDVLKKCQQAEKILKEQERLAYINPDLALEEKNKGNECFQKGDYPQAMKHYTEAIKRNPKDAKLYSNRAACYTKLLEFQLALKDCEECIQLEPTFIKGYTRKAAALEAMKDYTKAMDVYQKALDLDSSCKEAADGYQRCMMAQYNRHDSPEDVKRRAMADPEVQQIMSDPAMRLILEQMQKDPQALSEHLKNPVIAQKIQKLMDVGLIAIR",
# CHIP_HUMAN
"MKGKEEKEGGARLGAGGGSPEKSPSAQELKEQGNRLFVGRKYPEAAACYGRAITRNPLVAVYYTNRALCYLKMQQHEQALADCRRALELDGQSVKAHFFLGQCQLEMESYDEAIANLQRAYSLAKEQRLNFGDDIPSALRIAKKKRWNSIEERRIHQESELHSYLSRLIAAERERELEECQRNHEGDEDDSHVRAQQACIEAKHDKYMADMDELFSQVDEKRKKRDIPDYLCGKISFELMREPCITPSGITYDRKDIEEHLQRVGHFDPVTRSPLTQEQLIPNLAMKEVIDAFISENGWVEDY",
# SGT1_HUMAN
"MAAAAAGTATSQRFFQSFSDALIDEDPQAALEELTKALEQKPDDAQYYCQRAYCHILLGNYCVAVADAKKSLELNPNNSTAMLRKGICEYHEKNYAAALETFTEGQKLDIETGFHRVGQAGLQLLTSSDPPALDSQSAGITGADANFSVWIKRCQEAQNGSESEVWTHQSKIKYDWYQTESQVVITLMIKNVQKNDVNVEFSEKELSALVKLPSGEDYNLKLELLHPIIPEQSTFKVLSTKIEIKLKKPEAVRWEKLEGQGDVPTPKQFVADVKNLYPSSSPYTRNWDKLVGEIKEEEKNEKLEGDAALNRLFQQIYSDGSDEVKRAMNKSFMESGGTVLSTNWSDVGKRKVEINPPDDMEWKKY",
# TCPA_HUMAN
"MEGPLSVFGDRSTGETIRSQNVMAAASIANIVKSSLGPVGLDKMLVDDIGDVTITNDGATILKLLEVEHPAAKVLCELADLQDKEVGDGTTSVVIIAAELLKNADELVKQKIHPTSVISGYRLACKEAVRYINENLIVNTDELGRDCLINAAKTSMSSKIIGINGDFFANMVVDAVLAIKYTDIRGQPRYPVNSVNILKAHGRSQMESMLISGYALNCVVGSQGMPKRIVNAKIACLDFSLQKTKMKLGVQVVITDPEKLDQIRQRESDITKERIQKILATGANVILTTGGIDDMCLKYFVEAGAMAVRRVLKRDLKRIAKASGATILSTLANLEGEETFEAAMLGQAEEVVQERICDDELILIKNTKARTSASIILRGANDFMCDEMERSLHDALCVVKRVLESKSVVPGGGAVEAALSIYLENYATSMGSREQLAIAEFARSLLVIPNTLAVNAAQDSTDLVAKLRAFHNEAQVNPERKNLKWIGLDLSNGKPRDNKQAGVFEPTIVKVKSLKFATEAAITILRIDDLIKLHPESKDDKHGSYEDAVHSGALND",
# TMX1_HUMAN
"MAPSGSLAVPLAVLVLLLWGAPWTHGRRSNVRVITDENWRELLEGDWMIEFYAPWCPACQNLQPEWESFAEWGEDLEVNIAKVDVTEQPGLSGRFIITALPTIYHCKDGEFRRYQGPRTKKDFINFISDKEWKSIEPVSSWFGPGSVLMSSMSALFQLSMWIRTCHNYFIEDLGLPVWGSYTVFALATLFSGLLLGLCMIFVADCLCPSKRRRPQPYPYPSKKLLSESAQPLKKVEEEQEADEEDVSEEEAESKEGTNKDFPQNAIRQRSLGPSLATDKS",
# TMX2_HUMAN
"MAVLAPLIALVYSVPRLSRWLAQPYYLLSALLSAAFLLVRKLPPLCHGLPTQREDGNPCDFDWREVEILMFLSAIVMMKNRRSITVEQHIGNIFMFSKVANTILFFRLDIRMGLLYITLCIVFLMTCKPPLYMGPEYIKYFNDKTIDEELERDKRVTWIVEFFANWSNDCQSFAPIYADLSLKYNCTGLNFGKVDVGRYTDVSTRYKVSTSPLTKQLPTLILFQGGKEAMRRPQIDKKGRAVSWTFSEENVIREFNLNELYQRAKKLSKAGDNIPEEQPVASTPTTVSDGENKKDK",
# TMX3_HUMAN
"MAAWKSWTALRLCATVVVLDMVVCKGFVEDLDESFKENRNDDIWLVDFYAPWCGHCKKLEPIWNEVGLEMKSIGSPVKVGKMDATSYSSIASEFGVRGYPTIKLLKGDLAYNYRGPRTKDDIIEFAHRVSGALIRPLPSQQMFEHMQKRHRVFFVYVGGESPLKEKYIDAASELIVYTYFFSASEEVVPEYVTLKEMPAVLVFKDETYFVYDEYEDGDLSSWINRERFQNYLAMDGFLLYELGDTGKLVALAVIDEKNTSVEHTRLKSIIQEVARDYRDLFHRDFQFGHMDGNDYINTLLMDELTVPTVVVLNTSNQQYFLLDRQIKNVEDMVQFINNILDGTVEAQGGDSILQRLKRIVFDAKSTIVSIFKSSPLMGCFLFGLPLGVISIMCYGIYTADTDGGYIEERYEVSKSENENQEQIEESKEQQEPSSGGSVVPTVQEPKDVLEKKKD",
# TOM34_HUMAN
"MAPKFPDSVEELRAAGNESFRNGQYAEASALYGRALRVLQAQGSSDPEEESVLYSNRAACHLKDGNCRDCIKDCTSALALVPFSIKPLLRRASAYEALEKYPMAYVDYKTVLQIDDNVTSAVEGINRMTRALMDSLGPEWRLKLPSIPLVPVSAQKRWNSLPSENHKEMAKSKSKETTATKNRVPSAGDVEKARVLKEEGNELVKKGNHKKAIEKYSESLLCSNLESATYSNRALCYLVLKQYTEAVKDCTEALKLDGKNVKAFYRRAQAHKALKDYKSSFADISNLLQIEPRNGPAQKLRQEVKQNLH",
# TOM70_HUMAN
"MAASKPVEAAVVAAAVPSSGSGVGGGGTAGPGTGGLPRWQLALAVGAPLLLGAGAIYLWSRQQRRREARGRGDASGLKRNSERKTPEGRASPAPGSGHPEGPGAHLDMNSLDRAQAAKNKGNKYFKAGKYEQAIQCYTEAISLCPTEKNVDLSTFYQNRAAAFEQLQKWKEVAQDCTKAVELNPKYVKALFRRAKAHEKLDNKKECLEDVTAVCILEGFQNQQSMLLADKVLKLLGKEKAKEKYKNREPLMPSPQFIKSYFSSFTDDIISQPMLKGEKSDEDKDKEGEALEVKENSGYLKAKQYMEEENYDKIISECSKEIDAEGKYMAEALLLRATFYLLIGNANAAKPDLDKVISLKEANVKLRANALIKRGSMYMQQQQPLLSTQDFNMAADIDPQNADVYHHRGQLKILLDQVEEAVADFDECIRLRPESALAQAQKCFALYRQAYTGNNSSQIQAAMKGFEEVIKKFPRCAEGYALYAQALTDQQQFGKADEMYDKCIDLEPDNATTYVHKGLLQLQWKQDLDRGLELISKAIEIDNKCDFAYETMGTIEVQRGNMEKAIDMFNKAINLAKSEMEMAHLYSLCDAAHAQTEVAKKYGLKPPTL",
# TRAP1_HUMAN
"MARELRALLLWGRRLRPLLRAPALAAVPGGKPILCPRRTTAQLGPRRNPAWSLQAGRLFSTQTAEDKEEPLHSIISSTESVQGSTSKHEFQAETKKLLDIVARSLYSEKEVFIRELISNASDALEKLRHKLVSDGQALPEMEIHLQTNAEKGTITIQDTGIGMTQEELVSNLGTIARSGSKAFLDALQNQAEASSKIIGQFGVGFYSAFMVADRVEVYSRSAAPGSLGYQWLSDGSGVFEIAEASGVRTGTKIIIHLKSDCKEFSSEARVRDVVTKYSNFVSFPLYLNGRRMNTLQAIWMMDPKDVREWQHEEFYRYVAQAHDKPRYTLHYKTDAPLNIRSIFYVPDMKPSMFDVSRELGSSVALYSRKVLIQTKATDILPKWLRFIRGVVDSEDIPLNLSRELLQESALIRKLRDVLQQRLIKFFIDQSKKDAEKYAKFFEDYGLFMREGIVTATEQEVKEDIAKLLRYESSALPSGQLTSLSEYASRMRAGTRNIYYLCAPNRHLAEHSPYYEAMKKKDTEVLFCFEQFDELTLLHLREFDKKKLISVETDIVVDHYKEEKFEDRSPAAECLSEKETEELMAWMRNVLGSRVTNVKVTLRLDTHPAMVTVLEMGAARHFLRMQQLAKTQEERAQLLQPTLEINPRHALIKKLNQLRASEPGLAQLLVDQIYENAMIAAGLVDDPRAMVGRLNELLVKALERH",
# TSC1_HUMAN
"MAQQANVGELLAMLDSPMLGVRDDVTAVFKENLNSDRGPMLVNTLVDYYLETSSQPALHILTTLQEPHDKHLLDRINEYVGKAATRLSILSLLGHVIRLQPSWKHKLSQAPLLPSLLKCLKMDTDVVVLTTGVLVLITMLPMIPQSGKQHLLDFFDIFGRLSSWCLKKPGHVAEVYLVHLHASVYALFHRLYGMYPCNFVSFLRSHYSMKENLETFEEVVKPMMEHVRIHPELVTGSKDHELDPRRWKRLETHDVVIECAKISLDPTEASYEDGYSVSHQISARFPHRSADVTTSPYADTQNSYGCATSTPYSTSRLMLLNMPGQLPQTLSSPSTRLITEPPQATLWSPSMVCGMTTPPTSPGNVPPDLSHPYSKVFGTTAGGKGTPLGTPATSPPPAPLCHSDDYVHISLPQATVTPPRKEERMDSARPCLHRQHHLLNDRGSEEPPGSKGSVTLSDLPGFLGDLASEEDSIEKDKEEAAISRELSEITTAEAEPVVPRGGFDSPFYRDSLPGSQRKTHSAASSSQGASVNPEPLHSSLDKLGPDTPKQAFTPIDLPCGSADESPAGDRECQTSLETSIFTPSPCKIPPPTRVGFGSGQPPPYDHLFEVALPKTAHHFVIRKTEELLKKAKGNTEEDGVPSTSPMEVLDRLIQQGADAHSKELNKLPLPSKSVDWTHFGGSPPSDEIRTLRDQLLLLHNQLLYERFKRQQHALRNRRLLRKVIKAAALEEHNAAMKDQLKLQEKDIQMWKVSLQKEQARYNQLQEQRDTMVTKLHSQIRQLQHDREEFYNQSQELQTKLEDCRNMIAELRIELKKANNKVCHTELLLSQVSQKLSNSESVQQQMEFLNRQLLVLGEVNELYLEQLQNKHSDTTKEVEMMKAAYRKELEKNRSHVLQQTQRLDTSQKRILELESHLAKKDHLLLEQKKYLEDVKLQARGQLQAAESRYEAQKRITQVFELEILDLYGRLEKDGLLKKLEEEKAEAAEAAEERLDCCNDGCSDSMVGHNEEASGHNGETKTPRPSSARGSSGSRGGGGSSSSSSELSTPEKPPHQRAGPFSSRWETTMGEASASIPTTVGSLPSSKSFLGMKARELFRNKSESQCDEDGMTSSLSESLKTELGKDLGVEAKIPLNLDGPHPSPPTPDSVGQLHIMDYNETHHEHS",
# TTC1_HUMAN
"MGEKSENCGVPEDLLNGLKVTDTQEAECAGPPVPDPKNQHSQSKLLRDDEAHLQEDQGEEECFHDCSASFEEEPGADKVENKSNEDVNSSELDEEYLIELEKNMSDEEKQKRREESTRLKEEGNEQFKKGDYIEAESSYSRALEMCPSCFQKERSILFSNRAAARMKQDKKEMAINDCSKAIQLNPSYIRAILRRAELYEKTDKLDEALEDYKSILEKDPSIHQAREACMRLPKQIEERNERLKEEMLGKLKDLGNLVLRPFGLSTENFQIKQDSSTGSYSINFVQNPNNNR",
# TTC36_HUMAN
"MGTPNDQAVLQAIFNPDTPFGDIVGLDLGEEAEKEEREEDEVFPQAQLEQSKALELQGVMAAEAGDLSTALERFGQAICLLPERASAYNNRAQARRLQGDVAGALEDLERAVELSGGRGRAARQSFVQRGLLARLQGRDDDARRDFERAARLGSPFARRQLVLLNPYAALCNRMLADMMGQLRRPRDSR",
# TTC4_HUMAN
"MEQPGQDPTSDDVMDSFLEKFQSQPYRGGFHEDQWEKEFEKVPLFMSRAPSEIDPRENPDLACLQSIIFDEERSPEEQAKTYKDEGNDYFKEKDYKKAVISYTEGLKKKCADPDLNAVLYTNRAAAQYYLGNFRSALNDVTAARKLKPCHLKAIIRGALCHLELKHFAEAVNWCDEGLQIDAKEKKLLEMRAKADKLKRIEQRDVRKANLKEKKERNQNEALLQAIKARNIRLSEAACEDEDSASEGLGELFLDGLSTENPHGARLSLDGQGRLSWPVLFLYPEYAQSDFISAFHEDSRFIDHLMVMFGETPSWDLEQKYCPDNLEVYFEDEDRAELYRVPAKSTLLQVLQHQRYFVKALTPAFLVCVGSSPFCKNFLRGRKVYQIR",
# TTC5_HUMAN
"MMADEEEEVKPILQKLQELVDQLYSFRDCYFETHSVEDAGRKQQDVQKEMEKTLQQMEEVVGSVQGKAQVLMLTGKALNVTPDYSPKAEELLSKAVKLEPELVEAWNQLGEVYWKKGDVAAAHTCFSGALTHCRNKVSLQNLSMVLRQLRTDTEDEHSHHVMDSVRQAKLAVQMDVHDGRSWYILGNSYLSLYFSTGQNPKISQQALSAYAQAEKVDRKASSNPDLHLNRATLHKYEESYGEALEGFSRAAALDPAWPEPRQREQQLLEFLDRLTSLLESKGKVKTKKLQSMLGSLRPAHLGPCSDGHYQSASGQKVTLELKPLSTLQPGVNSGAVILGKVVFSLTTEEKVPFTFGLVDSDGPCYAVMVYNIVQSWGVLIGDSVAIPEPNLRLHRIQHKGKDYSFSSVRVETPLLLVVNGKPQGSSSQAVATVASRPQCE",
# TXD11_HUMAN
"MSECGGRGGGSSSSEDAEDEGGGGGGPAGSDCLSSSPTLATASSAGRLRRGLRGAFLMARQRPELLCGAVALGCALLLALKFTCSRAKDVIIPAKPPVSFFSLRSPVLDLFQGQLDYAEYVRRDSEVVLLFFYAPWCGQSIAARAEIEQAASRLSDQVLFVAINCWWNQGKCRKQKHFFYFPVIYLYHRSFGPIEYKGPMSAVYIEKFVRRVMKPLLYIPSQSELLDFLSNYEPGVLGYFEFSGSPQPPGYLTFFTSALHSLKKALESTSSPRALVSFTGEWHLETKIYVLDYLGTVRFGVITNKHLAKLVSLVHSGSVYLHRHFNTSLVFPREVLNYTAENICKWALENQETLFRWLRPHGGKSLLLNNELKKGPALFLFIPFNPLAESHPLIDEITEVALEYNNCHGDQVVERLLQHLRRVDAPVLESLALEVPAQLPDPPTITASPCCNTVVLPQWHSFSRTHNVCELCVNQTSGGMKPSSVSVPQCSFFEMAAALDSFYLKEQTFYHVASDSIECSNFLTSYSPFSYYTACCRTISRGVSGFIDSEQGVFEAPTVAFSSLEKKCEVDAPSSVPHIEENRYLFPEVDMTSTNFTGLSCRTNKTLNIYLLDSNLFWLYAERLGAPSSTQVKEFAAIVDVKEESHYILDPKQALMKLTLESFIQNFSVLYSPLKRHLIGSGSAQFPSQHLITEVTTDTFWEVVLQKQDVLLLYYAPWCGFCPSLNHIFIQLARNLPMDTFTVARIDVSQNDLPWEFMVDRLPTVLFFPCNRKDLSVKYPEDVPITLPNLLRFILHHSDPASSPQNVANSPTKECLQSEAVLQRGHISHLEREIQKLRAEISSLQRAQVQVESQLSSARRDEHRLRQQQRALEEQHSLLHAHSEQLQALYEQKTRELQELARKLQELADASENLLTENTWLKILVATMERKLEGRDGAESLAAQREVHPKQPEPSATPQLPGSSPPPANVSATLVSERNKENRTD",
# TXD12_HUMAN
"METRPRLGATCLLGFSFLLLVISSDGHNGLGKGFGDHIHWRTLEDGKKEAAASGLPLMVIIHKSWCGACKALKPKFAESTEISELSHNFVMVNLEDEEEPKDEDFSPDGGYIPRILFLDPSGKVHPEIINENGNPSYKYFYVSAEQVVQGMKEAQERLTGDAFRKKHLEDEL",
# TXND5_HUMAN
"MPARPGRLLPLLARPAALTALLLLLLGHGGGGRWGARAQEAAAAAADGPPAADGEDGQDPHSKHLYTADMFTHGIQSAAHFVMFFAPWCGHCQRLQPTWNDLGDKYNSMEDAKVYVAKVDCTAHSDVCSAQGVRGYPTLKLFKPGQEAVKYQGPRDFQTLENWMLQTLNEEPVTPEPEVEPPSAPELKQGLYELSASNFELHVAQGDHFIKFFAPWCGHCKALAPTWEQLALGLEHSETVKIGKVDCTQHYELCSGNQVRGYPTLLWFRDGKKVDQYKGKRDLESLREYVESQLQRTETGATETVTPSEAPVLAAEPEADKGTVLALTENNFDDTIAEGITFIKFYAPWCGHCKTLAPTWEELSKKEFPGLAGVKIAEVDCTAERNICSKYSVRGYPTLLLFRGGKKVSEHSGGRDLDSLHRFVLSQAKDEL",
# TRXR1_HUMAN
"MGCAEGKAVAAAAPTELQTKGKNGDGRRRSAKDHHPGKTLPENPAGFTSTATADSRALLQAYIDGHSVVIFSRSTCTRCTEVKKLFKSLCVPYFVLELDQTEDGRALEGTLSELAAETDLPVVFVKQRKIGGHGPTLKAYQEGRLQKLLKMNGPEDLPKSYDYDLIIIGGGSGGLAAAKEAAQYGKKVMVLDFVTPTPLGTRWGLGGTCVNVGCIPKKLMHQAALLGQALQDSRNYGWKVEETVKHDWDRMIEAVQNHIGSLNWGYRVALREKKVVYENAYGQFIGPHRIKATNNKGKEKIYSAERFLIATGERPRYLGIPGDKEYCISSDDLFSLPYCPGKTLVVGASYVALECAGFLAGIGLDVTVMVRSILLRGFDQDMANKIGEHMEEHGIKFIRQFVPIKVEQIEAGTPGRLRVVAQSTNSEEIIEGEYNTVMLAIGRDACTRKIGLETVGVKINEKTGKIPVTDEEQTNVPYIYAIGDILEDKVELTPVAIQAGRLLAQRLYAGSTVKCDYENVPTTVFTPLEYGACGLSEEKAVEKFGEENIEVYHSYFWPLEWTIPSRDNNKCYAKIICNTKDNERVVGFHVLGPNAGEVTQGFAAALKCGLTKKQLDSTIGIHPVCAEVFTTLSVTKRSGASILQAGCUG",
# UN45A_HUMAN
"MTVSGPGTPEPRPATPGASSVEQLRKEGNELFKCGDYGGALAAYTQALGLDATPQDQAVLHRNRAACHLKLEDYDKAETEASKAIEKDGGDVKALYRRSQALEKLGRLDQAVLDLQRCVSLEPKNKVFQEALRNIGGQIQEKVRYMSSTDAKVEQMFQILLDPEEKGTEKKQKASQNLVVLAREDAGAEKIFRSNGVQLLQRLLDMGETDLMLAALRTLVGICSEHQSRTVATLSILGTRRVVSILGVESQAVSLAACHLLQVMFDALKEGVKKGFRGKEGAIIVDPARELKVLISNLLDLLTEVGVSGQGRDNALTLLIKAVPRKSLKDPNNSLTLWVIDQGLKKILEVGGSLQDPPGELAVTANSRMSASILLSKLFDDLKCDAERENFHRLCENYIKSWFEGQGLAGKLRAIQTVSCLLQGPCDAGNRALELSGVMESVIALCASEQEEEQLVAVEALIHAAGKAKRASFITANGVSLLKDLYKCSEKDSIRIRALVGLCKLGSAGGTDFSMKQFAEGSTLKLAKQCRKWLCNDQIDAGTRRWAVEGLAYLTFDADVKEEFVEDAAALKALFQLSRLEERSVLFAVASALVNCTNSYDYEEPDPKMVELAKYAKQHVPEQHPKDKPSFVRARVKKLLAAGVVSAMVCMVKTESPVLTSSCRELLSRVFLALVEEVEDRGTVVAQGGGRALIPLALEGTDVGQTKAAQALAKLTITSNPEMTFPGERIYEVVRPLVSLLHLNCSGLQNFEALMALTNLAGISERLRQKILKEKAVPMIEGYMFEEHEMIRRAATECMCNLAMSKEVQDLFEAQGNDRLKLLVLYSGEDDELLQRAAAGGLAMLTSMRPTLCSRIPQVTTHWLEILQALLLSSNQELQHRGAVVVLNMVEASREIASTLMESEMMEILSVLAKGDHSPVTRAAAACLDKAVEYGLIQPNQDGE",
# UN45B_HUMAN
"MAEVEAVQLKEEGNRHFQLQDYKAATNSYSQALKLTKDKALLATLYRNRAACGLKTESYVQAASDASRAIDINSSDIKALYRRCQALEHLGKLDQAFKDVQRCATLEPRNQNFQEMLRRLNTSIQEKLRVQFSTDSRVQKMFEILLDENSEADKREKAANNLIVLGREEAGAEKIFQNNGVALLLQLLDTKKPELVLAAVRTLSGMCSGHQARATVILHAVRIDRICSLMAVENEEMSLAVCNLLQAIIDSLSGEDKREHRGKEEALVLDTKKDLKQITSHLLDMLVSKKVSGQGRDQALNLLNKNVPRKDLAIHDNSRTIYVVDNGLRKILKVVGQVPDLPSCLPLTDNTRMLASILINKLYDDLRCDPERDHFRKICEEYITGKFDPQDMDKNLNAIQTVSGILQGPFDLGNQLLGLKGVMEMMVALCGSERETDQLVAVEALIHASTKLSRATFIITNGVSLLKQIYKTTKNEKIKIRTLVGLCKLGSAGGTDYGLRQFAEGSTEKLAKQCRKWLCNMSIDTRTRRWAVEGLAYLTLDADVKDDFVQDVPALQAMFELAKAGTSDKTILYSVATTLVNCTNSYDVKEVIPELVQLAKFSKQHVPEEHPKDKKDFIDMRVKRLLKAGVISALACMVKADSAILTDQTKELLARVFLALCDNPKDRGTIVAQGGGKALIPLALEGTDVGKVKAAHALAKIAAVSNPDIAFPGERVYEVVRPLVRLLDTQRDGLQNYEALLGLTNLSGRSDKLRQKIFKERALPDIENYMFENHDQLRQAATECMCNMVLHKEVQERFLADGNDRLKLVVLLCGEDDDKVQNAAAGALAMLTAAHKKLCLKMTQVTTQWLEILQRLCLHDQLSVQHRGLVIAYNLLAADAELAKKLVESELLEILTVVGKQEPDEKKAEVVQTARECLIKCMDYGFIKPVS",
# RMP_HUMAN
"MEAPTVETPPDPSPPSAPAPALVPLRAPDVARLREEQEKVVTNCQERIQHWKKVDNDYNALRERLSTLPDKLSYNIMVPFGPFAFMPGKLVHTNEVTVLLGDNWFAKCSAKQAVGLVEHRKEHVRKTIDDLKKVMKNFESRVEFTEDLQKMSDAAGDIVDIREEIKCDFEFKAKHRIAHKPHSKPKTSDIFEADIANDVKSKDLLADKELWARLEELERQEELLGELDSKPDTVIANGEDTTSSEEEKEDRNTNVNAMHQVTDSHTPCHKDVASSEPFSGQVNSQLNCSVNGSSSYHSDDDDDDDDDDDDDNIDDDDGDNDHEALGVGDNSIPTIYFSHTVEPKRVRINTGKNTTLKFSEKKEEAKRKRKNSTGSGHSAQELPTIRTPADIYRAFVDVVNGEYVPRKSILKSRSRENSVCSDTSESSAAEFDDRRGVLRSISCEEATCSDTSESILEEEPQENQKKLLPLSVTPEAFSGTVIEKEFVSPSLTPPPAIAHPALPTIPERKEVLLEASEETGKRVSKFKAARLQQKD",
# UXT_HUMAN
"MATPPKRRAVEATGEKVLRYETFISDVLQRDLRKVLDHRDKVYEQLAKYLQLRNVIERLQEAKHSELYMQVDLGCNFFVDTVVPDTSRIYVALGYGFFLELTLAEALKFIDRKSSLLTELSNSLTKDSMNIKAHIHMLLEGLRELQGLQNFPEKPHH",
# PFD3_HUMAN
"MAAVKDSCGKGEMATGNGRRLHLGIPEAVFVEDVDSFMKQPGNETADTVLKKLDEQYQKYKFMELNLAQKKRRLKGQIPEIKQTLEILKYMQKKKESTNSMETRFLLADNLYCKASVPPTDKVCLWLGANVMLEYDIDEAQALLEKNLSTATKNLDSLEEDLDFLRDQFTTTEVNMARVYNWDVKRRNKDDSTKNKA",
# TERA_HUMAN
"MASGADSKGDDLSTAILKQKNRPNRLIVDEAINEDNSVVSLSQPKMDELQLFRGDTVLLKGKKRREAVCIVLSDDTCSDEKIRMNRVVRNNLRVRLGDVISIQPCPDVKYGKRIHVLPIDDTVEGITGNLFEVYLKPYFLEAYRPIRKGDIFLVRGGMRAVEFKVVETDPSPYCIVAPDTVIHCEGEPIKREDEEESLNEVGYDDIGGCRKQLAQIKEMVELPLRHPALFKAIGVKPPRGILLYGPPGTGKTLIARAVANETGAFFFLINGPEIMSKLAGESESNLRKAFEEAEKNAPAIIFIDELDAIAPKREKTHGEVERRIVSQLLTLMDGLKQRAHVIVMAATNRPNSIDPALRRFGRFDREVDIGIPDATGRLEILQIHTKNMKLADDVDLEQVANETHGHVGADLAALCSEAALQAIRKKMDLIDLEDETIDAEVMNSLAVTMDDFRWALSQSNPSALRETVVEVPQVTWEDIGGLEDVKRELQELVQYPVEHPDKFLKFGMTPSKGVLFYGPPGCGKTLLAKAIANECQANFISIKGPELLTMWFGESEANVREIFDKARQAAPCVLFFDELDSIAKARGGNIGDGGGAADRVINQILTEMDGMSTKKNVFIIGATNRPDIIDPAILRPGRLDQLIYIPLPDEKSRVAILKANLRKSPVAKDVDLEFLAKMTNGFSGADLTEICQRACKLAIRESIESEIRRERERQTNPSAMEVEEDDPVPEIRRDHFEEAMRFARRSVSDNDIRKYEMFAQTLQQSRGFGSFRFPSGNQGGAGPSQGSGGGTGGSVYTEDNDDDLYG"
]


# Add protein names corresponding to the sequences
protein_names = [
"AHSA1_HUMAN",
"AHSA2_HUMAN",
"AIP_HUMAN",
"AIPL1_HUMAN",
"BAG1_HUMAN",
"BAG2_HUMAN",
"BAG3_HUMAN",
"BAG4_HUMAN",
"BAG5_HUMAN",
"BAG6_HUMAN",
"BBS10_HUMAN",
"BBS12_HUMAN",
"BCS1_HUMAN",
"CALR_HUMAN",
"CALX_HUMAN",
"TCPB_HUMAN",
"TCPG_HUMAN",
"TCPD_HUMAN",
"TCPE_HUMAN",
"TCPZ_HUMAN",
"TCPW_HUMAN",
"TCPH_HUMAN",
"TCPQ_HUMAN",
"TCPQM_HUMAN",
"CDC37_HUMAN",
"CD37L_HUMAN",
"CHRD1_HUMAN",
"CLGN_HUMAN",
"CLPB_HUMAN",
"CLPX_HUMAN",
"CRYAA_HUMAN",
"CRYAB_HUMAN",
"DNJA1_HUMAN",
"DNJA2_HUMAN",
"DNJA3_HUMAN",
"DNJA4_HUMAN",
"DNJB1_HUMAN",
"DJB11_HUMAN",
"DJB12_HUMAN",
"DJB13_HUMAN",
"DJB14_HUMAN",
"DNJB2_HUMAN",
"DNJB3_HUMAN",
"DNJB4_HUMAN",
"DNJB5_HUMAN",
"DNJB6_HUMAN",
"DNJB7_HUMAN",
"DNJB8_HUMAN",
"DNJB9_HUMAN",
"DNJC1_HUMAN",
"DJC10_HUMAN",
"DJC11_HUMAN",
"DJC12_HUMAN",
"DJC13_HUMAN",
"DJC14_HUMAN",
"DJC15_HUMAN",
"DJC16_HUMAN",
"DJC17_HUMAN",
"DJC18_HUMAN",
"TIM14_HUMAN",
"DNJC2_HUMAN",
"DJC21_HUMAN",
"DJC22_HUMAN",
"DJC24_HUMAN",
"DJC25_HUMAN",
"DJC27_HUMAN",
"DJC28_HUMAN",
"DNJC3_HUMAN",
"DJC30_HUMAN",
"DNJC4_HUMAN",
"DNJC5_HUMAN",
"DNJ5B_HUMAN",
"DNJ5G_HUMAN",
"AUXI_HUMAN",
"DNJC7_HUMAN",
"DNJC8_HUMAN",
"DNJC9_HUMAN",
"DAAF4_HUMAN",
"EDEM1_HUMAN",
"EDEM2_HUMAN",
"EDEM3_HUMAN",
"ERO1A_HUMAN",
"ERO1B_HUMAN",
"ERP44_HUMAN",
"FKB10_HUMAN",
"FKB11_HUMAN",
"FKB14_HUMAN",
"FKB15_HUMAN",
"FKB1A_HUMAN",
"FKB1B_HUMAN",
"FKBP2_HUMAN",
"FKBP3_HUMAN",
"FKBP4_HUMAN",
"FKBP5_HUMAN",
"FKBP6_HUMAN",
"FKBP7_HUMAN",
"FKBP8_HUMAN",
"FKBP9_HUMAN",
"FKBPL_HUMAN",
"GAK_HUMAN",
"GRPE1_HUMAN",
"GRPE2_HUMAN",
"HSC20_HUMAN",
"HS90A_HUMAN",
"HS90B_HUMAN",
"ENPL_HUMAN",
"HS12A_HUMAN",
"HS12B_HUMAN",
"HSP13_HUMAN",
"HSP7E_HUMAN",
"HS71A_HUMAN",
"HS71B_HUMAN",
"HS71L_HUMAN",
"HSP72_HUMAN",
"HSP74_HUMAN",
"HS74L_HUMAN",
"BIP_HUMAN",
"HSP76_HUMAN",
"HSP77_HUMAN",
"HSP7C_HUMAN",
"HSPA9_HUMAN",
"HSPB1_HUMAN",
"IFT25_HUMAN",
"HSPB2_HUMAN",
"HSPB3_HUMAN",
"HSPB6_HUMAN",
"HSPB7_HUMAN",
"HSPB8_HUMAN",
"HSPB9_HUMAN",
"HPBP1_HUMAN",
"CH60_HUMAN",
"CH10_HUMAN",
"HS105_HUMAN",
"HYOU1_HUMAN",
"MKKS_HUMAN",
"MZB1_HUMAN",
"NUDC_HUMAN",
"ODFP1_HUMAN",
"PDIA1_HUMAN",
"PDIA2_HUMAN",
"PDIA3_HUMAN",
"PDIA4_HUMAN",
"PDIA5_HUMAN",
"PDIA6_HUMAN",
"PDRG1_HUMAN",
"PFD1_HUMAN",
"PFD2_HUMAN",
"PFD4_HUMAN",
"PFD5_HUMAN",
"PFD6_HUMAN",
"PHB1_HUMAN",
"PHB2_HUMAN",
"PIHD1_HUMAN",
"PPIA_HUMAN",
"PAL4A_HUMAN",
"PPIB_HUMAN",
"PPIC_HUMAN",
"PPID_HUMAN",
"PPP5_HUMAN",
"TEBP_HUMAN",
"RPAP3_HUMAN",
"RUVB1_HUMAN",
"RUVB2_HUMAN",
"SACS_HUMAN",
"SEC63_HUMAN",
"SERPH_HUMAN",
"SGTA_HUMAN",
"SGTB_HUMAN",
"SIL1_HUMAN",
"F10A1_HUMAN",
"STIP1_HUMAN",
"CHIP_HUMAN",
"SGT1_HUMAN",
"TCPA_HUMAN",
"TMX1_HUMAN",
"TMX2_HUMAN",
"TMX3_HUMAN",
"TOM34_HUMAN",
"TOM70_HUMAN",
"TRAP1_HUMAN",
"TSC1_HUMAN",
"TTC1_HUMAN",
"TTC36_HUMAN",
"TTC4_HUMAN",
"TTC5_HUMAN",
"TXD11_HUMAN",
"TXD12_HUMAN",
"TXND5_HUMAN",
"TRXR1_HUMAN",
"UN45A_HUMAN",
"UN45B_HUMAN",
"RMP_HUMAN",
"UXT_HUMAN",
"PFD3_HUMAN",
"TERA_HUMAN"
]

In [21]:
#Glycolysis
sequences = [
# ADH1A_HUMAN
"MSTAGKVIKCKAAVLWELKKPFSIEEVEVAPPKAHEVRIKMVAVGICGTDDHVVSGTMVTPLPVILGHEAAGIVESVGEGVTTVKPGDKVIPLAIPQCGKCRICKNPESNYCLKNDVSNPQGTLQDGTSRFTCRRKPIHHFLGISTFSQYTVVDENAVAKIDAASPLEKVCLIGCGFSTGYGSAVNVAKVTPGSTCAVFGLGGVGLSAIMGCKAAGAARIIAVDINKDKFAKAKELGATECINPQDYKKPIQEVLKEMTDGGVDFSFEVIGRLDTMMASLLCCHEACGTSVIVGVPPDSQNLSMNPMLLLTGRTWKGAILGGFKSKECVPKLVADFMAKKFSLDALITHVLPFEKINEGFDLLHSGKSIRTILMF",
# ADH4_HUMAN
"MGTKGKVIKCKAAIAWEAGKPLCIEEVEVAPPKAHEVRIQIIATSLCHTDATVIDSKFEGLAFPVIVGHEAAGIVESIGPGVTNVKPGDKVIPLYAPLCRKCKFCLSPLTNLCGKISNLKSPASDQQLMEDKTSRFTCKGKPVYHFFGTSTFSQYTVVSDINLAKIDDDANLERVCLLGCGFSTGYGAAINNAKVTPGSTCAVFGLGGVGLSAVMGCKAAGASRIIGIDINSEKFVKAKALGATDCLNPRDLHKPIQEVIIELTKGGVDFALDCAGGSETMKAALDCTTAGWGSCTFIGVAAGSKGLTIFPEELIIGRTINGTFFGGWKSVDSIPKLVTDYKNKKFNLDALVTHTLPFDKISEAFDLMNQGKSVRTILIF",
# ADH6_HUMAN
"MSTTGQVIRCKAAILWKPGAPFSIEEVEVAPPKAKEVRIKVVATGLCGTEMKVLGSKHLDLLYPTILGHEGAGIVESIGEGVSTVKPGDKVITLFLPQCGECTSCLNSEGNFCIQFKQSKTQLMSDGTSRFTCKGKSIYHFGNTSTFCEYTVIKEISVAKIDAVAPLEKVCLISCGFSTGFGAAINTAKVTPGSTCAVFGLGGVGLSVVMGCKAAGAARIIGVDVNKEKFKKAQELGATECLNPQDLKKPIQEVLFDMTDAGIDFCFEAIGNLDVLAAALASCNESYGVCVVVGVLPASVQLKISGQLFFSGRSLKGSVFGGWKSRQHIPKLVADYMAEKLNLDPLITHTLNLDKINEAVELMKTGKW",
# ADH7_HUMAN
"MFAEIQIQDKDRMGTAGKVIKCKAAVLWEQKQPFSIEEIEVAPPKTKEVRIKILATGICRTDDHVIKGTMVSKFPVIVGHEATGIVESIGEGVTTVKPGDKVIPLFLPQCRECNACRNPDGNLCIRSDITGRGVLADGTTRFTCKGKPVHHFMNTSTFTEYTVVDESSVAKIDDAAPPEKVCLIGCGFSTGYGAAVKTGKVKPGSTCVVFGLGGVGLSVIMGCKSAGASRIIGIDLNKDKFEKAMAVGATECISPKDSTKPISEVLSEMTGNNVGYTFEVIGHLETMIDALASCHMNYGTSVVVGVPPSAKMLTYDPMLLFTGRTWKGCVFGGLKSRDDVPKLVTEFLAKKFDLDQLITHVLPFKKISEGFELLNSGQSIRTVLTF",
# HOT_HUMAN
"MAAAARARVAYLLRQLQRAACQCPTHSHTYSQAPGLSPSGKTTDYAFEMAVSNIRYGAAVTKEVGMDLKNMGAKNVCLMTDKNLSKLPPVQVAMDSLVKNGIPFTVYDNVRVEPTDSSFMEAIEFAQKGAFDAYVAVGGGSTMDTCKAANLYASSPHSDFLDYVSAPIGKGKPVSVPLKPLIAVPTTSGTGSETTGVAIFDYEHLKVKIGITSRAIKPTLGLIDPLHTLHMPARVVANSGFDVLCHALESYTTLPYHLRSPCPSNPITRPAYQGSNPISDIWAIHALRIVAKYLKRAVRNPDDLEARSHMHLASAFAGIGFGNAGVHLCHGMSYPISGLVKMYKAKDYNVDHPLVPHGLSVVLTSPAVFTFTAQMFPERHLEMAEILGADTRTARIQDAGLVLADTLRKFLFDLDVDDGLAAVGYSKADIPALVKGTLPQERVTKLAPCPQSEEDLAALFEASMKLY",
# AK1A1_HUMAN
"MAASCVLLHTGQKMPLIGLGTWKSEPGQVKAAVKYALSVGYRHIDCAAIYGNEPEIGEALKEDVGPGKAVPREELFVTSKLWNTKHHPEDVEPALRKTLADLQLEYLDLYLMHWPYAFERGDNPFPKNADGTICYDSTHYKETWKALEALVAKGLVQALGLSNFNSRQIDDILSVASVRPAVLQVECHPYLAQNELIAHCQARGLEVTAYSPLGSSDRAWRDPDEPVLLEEPVVLALAEKYGRSPAQILLRWQVQRKVICIPKSITPSRILQNIKVFDFTFSPEEMKQLNALNKNWRYIVPMLTVDGKRVPRDAGHPLYPFNDPY",
# AL1A1_HUMAN
"MSSSGTPDLPVLLTDLKIQYTKIFINNEWHDSVSGKKFPVFNPATEEELCQVEEGDKEDVDKAVKAARQAFQIGSPWRTMDASERGRLLYKLADLIERDRLLLATMESMNGGKLYSNAYLNDLAGCIKTLRYCAGWADKIQGRTIPIDGNFFTYTRHEPIGVCGQIIPWNFPLVMLIWKIGPALSCGNTVVVKPAEQTPLTALHVASLIKEAGFPPGVVNIVPGYGPTAGAAISSHMDIDKVAFTGSTEVGKLIKEAAGKSNLKRVTLELGGKSPCIVLADADLDNAVEFAHHGVFYHQGQCCIAASRIFVEESIYDEFVRRSVERAKKYILGNPLTPGVTQGPQIDKEQYDKILDLIESGKKEGAKLECGGGPWGNKGYFVQPTVFSNVTDEMRIAKEEIFGPVQQIMKFKSLDDVIKRANNTFYGLSAGVFTKDIDKAITISSALQAGTVWVNCYGVVSAQCPFGGFKMSGNGRELGEYGFHEYTEVKTVTVKISQKNS",
# AL1A2_HUMAN
"MTSSKIEMPGEVKADPAALMASLHLLPSPTPNLEIKYTKIFINNEWQNSESGRVFPVYNPATGEQVCEVQEADKADIDKAVQAARLAFSLGSVWRRMDASERGRLLDKLADLVERDRAVLATMESLNGGKPFLQAFYVDLQGVIKTFRYYAGWADKIHGMTIPVDGDYFTFTRHEPIGVCGQIIPWNFPLLMFAWKIAPALCCGNTVVIKPAEQTPLSALYMGALIKEAGFPPGVINILPGYGPTAGAAIASHIGIDKIAFTGSTEVGKLIQEAAGRSNLKRVTLELGGKSPNIIFADADLDYAVEQAHQGVFFNQGQCCTAGSRIFVEESIYEEFVRRSVERAKRRVVGSPFDPTTEQGPQIDKKQYNKILELIQSGVAEGAKLECGGKGLGRKGFFIEPTVFSNVTDDMRIAKEEIFGPVQEILRFKTMDEVIERANNSDFGLVAAVFTNDINKALTVSSAMQAGTVWINCYNALNAQSPFGGFKMSGNGREMGEFGLREYSEVKTVTVKIPQKNS",
# AL1B1_HUMAN
"MLRFLAPRLLSLQGRTARYSSAAALPSPILNPDIPYNQLFINNEWQDAVSKKTFPTVNPTTGEVIGHVAEGDRADVDRAVKAAREAFRLGSPWRRMDASERGRLLNRLADLVERDRVYLASLETLDNGKPFQESYALDLDEVIKVYRYFAGWADKWHGKTIPMDGQHFCFTRHEPVGVCGQIIPWNFPLVMQGWKLAPALATGNTVVMKVAEQTPLSALYLASLIKEAGFPPGVVNIITGYGPTAGAAIAQHVDVDKVAFTGSTEVGHLIQKAAGDSNLKRVTLELGGKSPSIVLADADMEHAVEQCHEALFFNMGQCCCAGSRTFVEESIYNEFLERTVEKAKQRKVGNPFELDTQQGPQVDKEQFERVLGYIQLGQKEGAKLLCGGERFGERGFFIKPTVFGGVQDDMRIAKEEIFGPVQPLFKFKKIEEVVERANNTRYGLAAAVFTRDLDKAMYFTQALQAGTVWVNTYNIVTCHTPFGGFKESGNGRELGEDGLKAYTEVKTVTIKVPQKNS",
# ALDH2_HUMAN
"MLRAAARFGPRLGRRLLSAAATQAVPAPNQQPEVFCNQIFINNEWHDAVSRKTFPTVNPSTGEVICQVAEGDKEDVDKAVKAARAAFQLGSPWRRMDASHRGRLLNRLADLIERDRTYLAALETLDNGKPYVISYLVDLDMVLKCLRYYAGWADKYHGKTIPIDGDFFSYTRHEPVGVCGQIIPWNFPLLMQAWKLGPALATGNVVVMKVAEQTPLTALYVANLIKEAGFPPGVVNIVPGFGPTAGAAIASHEDVDKVAFTGSTEIGRVIQVAAGSSNLKRVTLELGGKSPNIIMSDADMDWAVEQAHFALFFNQGQCCCAGSRTFVQEDIYDEFVERSVARAKSRVVGNPFDSKTEQGPQVDETQFKKILGYINTGKQEGAKLLCGGGIAADRGYFIQPTVFGDVQDGMTIAKEEIFGPVMQILKFKTIEEVVGRANNSTYGLAAAVFTKDLDKANYLSQALQAGTVWVNCYDVFGAQSPFGGYKMSGSGRELGEYGLQAYTEVKTVTVKVPQKNS",
# AL3A1_HUMAN
"MSKISEAVKRARAAFSSGRTRPLQFRIQQLEALQRLIQEQEQELVGALAADLHKNEWNAYYEEVVYVLEEIEYMIQKLPEWAADEPVEKTPQTQQDELYIHSEPLGVVLVIGTWNYPFNLTIQPMVGAIAAGNSVVLKPSELSENMASLLATIIPQYLDKDLYPVINGGVPETTELLKERFDHILYTGSTGVGKIIMTAAAKHLTPVTLELGGKSPCYVDKNCDLDVACRRIAWGKFMNSGQTCVAPDYILCDPSIQNQIVEKLKKSLKEFYGEDAKKSRDYGRIISARHFQRVMGLIEGQKVAYGGTGDAATRYIAPTILTDVDPQSPVMQEEIFGPVLPIVCVRSLEEAIQFINQREKPLALYMFSSNDKVIKKMIAETSSGGVAANDVIVHITLHSLPFGGVGNSGMGSYHGKKSFETFSHRRSCLVRPLMNDEGLKVRYPPSPAKMTQH",
# AL3A2_HUMAN
"MELEVRRVRQAFLSGRSRPLRFRLQQLEALRRMVQEREKDILTAIAADLCKSEFNVYSQEVITVLGEIDFMLENLPEWVTAKPVKKNVLTMLDEAYIQPQPLGVVLIIGAWNYPFVLTIQPLIGAIAAGNAVIIKPSELSENTAKILAKLLPQYLDQDLYIVINGGVEETTELLKQRFDHIFYTGNTAVGKIVMEAAAKHLTPVTLELGGKSPCYIDKDCDLDIVCRRITWGKYMNCGQTCIAPDYILCEASLQNQIVWKIKETVKEFYGENIKESPDYERIINLRHFKRILSLLEGQKIAFGGETDEATRYIAPTVLTDVDPKTKVMQEEIFGPILPIVPVKNVDEAINFINEREKPLALYVFSHNHKLIKRMIDETSSGGVTGNDVIMHFTLNSFPFGGVGSSGMGAYHGKHSFDTFSHQRPCLLKSLKREGANKLRYPPNSQSKVDWGKFFLLKRFNKEKLGLLLLTFLGIVAAVLVKAEYY",
# AL3B1_HUMAN
"MDPLGDTLRRLREAFHAGRTRPAEFRAAQLQGLGRFLQENKQLLHDALAQDLHKSAFESEVSEVAISQGEVTLALRNLRAWMKDERVPKNLATQLDSAFIRKEPFGLVLIIAPWNYPLNLTLVPLVGALAAGNCVVLKPSEISKNVEKILAEVLPQYVDQSCFAVVLGGPQETGQLLEHRFDYIFFTGSPRVGKIVMTAAAKHLTPVTLELGGKNPCYVDDNCDPQTVANRVAWFRYFNAGQTCVAPDYVLCSPEMQERLLPALQSTITRFYGDDPQSSPNLGRIINQKQFQRLRALLGCGRVAIGGQSDESDRYIAPTVLVDVQEMEPVMQEEIFGPILPIVNVQSLDEAIEFINRREKPLALYAFSNSSQVVKRVLTQTSSGGFCGNDGFMHMTLASLPFGGVGASGMGRYHGKFSFDTFSHHRACLLRSPGMEKLNALRYPPQSPRRLRMLLVAMEAQGCSCTLL",
# AL9A1_HUMAN
"MSTGTFVVSQPLNYRGGARVEPADASGTEKAFEPATGRVIATFTCSGEKEVNLAVQNAKAAFKIWSQKSGMERCRILLEAARIIREREDEIATMECINNGKSIFEARLDIDISWQCLEYYAGLAASMAGEHIQLPGGSFGYTRREPLGVCVGIGAWNYPFQIASWKSAPALACGNAMVFKPSPFTPVSALLLAEIYSEAGVPPGLFNVVQGGAATGQFLCQHPDVAKVSFTGSVPTGMKIMEMSAKGIKPVTLELGGKSPLIIFSDCDMNNAVKGALMANFLTQGQVCCNGTRVFVQKEILDKFTEEVVKQTQRIKIGDPLLEDTRMGPLINRPHLERVLGFVKVAKEQGAKVLCGGDIYVPEDPKLKDGYYMRPCVLTNCRDDMTCVKEEIFGPVMSILSFDTEAEVLERANDTTFGLAAGVFTRDIQRAHRVVAELQAGTCFINNYNVSPVELPFGGYKKSGFGRENGRVTIEYYSQLKTVCVEMGDVESAF",
# ALDOA_HUMAN
"MPYQYPALTPEQKKELSDIAHRIVAPGKGILAADESTGSIAKRLQSIGTENTEENRRFYRQLLLTADDRVNPCIGGVILFHETLYQKADDGRPFPQVIKSKGGVVGIKVDKGVVPLAGTNGETTTQGLDGLSERCAQYKKDGADFAKWRCVLKIGEHTPSALAIMENANVLARYASICQQNGIVPIVEPEILPDGDHDLKRCQYVTEKVLAAVYKALSDHHIYLEGTLLKPNMVTPGHACTQKFSHEEIAMATVTALRRTVPPAVTGITFLSGGQSEEEASINLNAINKCPLLKPWALTFSYGRALQASALKAWGGKKENLKAAQEEYVKRALANSLACQGKYTPSGQAGAAASESLFVSNHAY",
# ALDOB_HUMAN
"MAHRFPALTQEQKKELSEIAQSIVANGKGILAADESVGTMGNRLQRIKVENTEENRRQFREILFSVDSSINQSIGGVILFHETLYQKDSQGKLFRNILKEKGIVVGIKLDQGGAPLAGTNKETTIQGLDGLSERCAQYKKDGVDFGKWRAVLRIADQCPSSLAIQENANALARYASICQQNGLVPIVEPEVIPDGDHDLEHCQYVTEKVLAAVYKALNDHHVYLEGTLLKPNMVTAGHACTKKYTPEQVAMATVTALHRTVPAAVPGICFLSGGMSEEDATLNLNAINLCPLPKPWKLSFSYGRALQASALAAWGGKAANKEATQEAFMKRAMANCQAAKGQYVHTGSSGAASTQSLFTACYTY",
# ALDOC_HUMAN
"MPHSYPALSAEQKKELSDIALRIVAPGKGILAADESVGSMAKRLSQIGVENTEENRRLYRQVLFSADDRVKKCIGGVIFFHETLYQKDDNGVPFVRTIQDKGIVVGIKVDKGVVPLAGTDGETTTQGLDGLSERCAQYKKDGADFAKWRCVLKISERTPSALAILENANVLARYASICQQNGIVPIVEPEILPDGDHDLKRCQYVTEKVLAAVYKALSDHHVYLEGTLLKPNMVTPGHACPIKYTPEEIAMATVTALRRTVPPAVPGVTFLSGGQSEEEASFNLNAINRCPLPRPWALTFSYGRALQASALNAWRGQRDNAGAATEEFIKRAEVNGLAAQGKYEGSGEDGGAAAQSLYIANHAY",
# PMGE_HUMAN
"MSKYKLIMLRHGEGAWNKENRFCSWVDQKLNSEGMEEARNCGKQLKALNFEFDLVFTSVLNRSIHTAWLILEELGQEWVPVESSWRLNERHYGALIGLNREQMALNHGEEQVRLWRRSYNVTPPPIEESHPYYQEIYNDRRYKVCDVPLDQLPRSESLKDVLERLLPYWNERIAPEVLRGKTILISAHGNSSRALLKHLEGISDEDIINITLPTGVPILLELDENLRAVGPHQFLGDQEAIQAAIKKVEDQGKVKQAKK",
# ODP2_HUMAN
"MWRVCARRAQNVAPWAGLEARWTALQEVPGTPRVTSRSGPAPARRNSVTTGYGGVRALCGWTPSSGATPRNRLLLQLLGSPGRRYYSLPPHQKVPLPSLSPTMQAGTIARWEKKEGDKINEGDLIAEVETDKATVGFESLEECYMAKILVAEGTRDVPIGAIICITVGKPEDIEAFKNYTLDSSAAPTPQAAPAPTPAATASPPTPSAQAPGSSYPPHMQVLLPALSPTMTMGTVQRWEKKVGEKLSEGDLLAEIETDKATIGFEVQEEGYLAKILVPEGTRDVPLGTPLCIIVEKEADISAFADYRPTEVTDLKPQVPPPTPPPVAAVPPTPQPLAPTPSAPCPATPAGPKGRVFVSPLAKKLAVEKGIDLTQVKGTGPDGRITKKDIDSFVPSKVAPAPAAVVPPTGPGMAPVPTGVFTDIPISNIRRVIAQRLMQSKQTIPHYYLSIDVNMGEVLLVRKELNKILEGRSKISVNDFIIKASALACLKVPEANSSWMDTVIRQNHVVDVSVAVSTPAGLITPIVFNAHIKGVETIANDVVSLATKAREGKLQPHEFQGGTFTISNLGMFGIKNFSAIINPPQACILAIGASEDKLVPADNEKGFDVASMMSVTLSCDHRVVDGAVGAQWLAEFRKYLEKPITMLL",
# DLDH_HUMAN
"MQSWSRVYCSLAKRGHFNRISHGLQGLSAVPLRTYADQPIDADVTVIGSGPGGYVAAIKAAQLGFKTVCIEKNETLGGTCLNVGCIPSKALLNNSHYYHMAHGKDFASRGIEMSEVRLNLDKMMEQKSTAVKALTGGIAHLFKQNKVVHVNGYGKITGKNQVTATKADGGTQVIDTKNILIATGSEVTPFPGITIDEDTIVSSTGALSLKKVPEKMVVIGAGVIGVELGSVWQRLGADVTAVEFLGHVGGVGIDMEISKNFQRILQKQGFKFKLNTKVTGATKKSDGKIDVSIEAASGGKAEVITCDVLLVCIGRRPFTKNLGLEELGIELDPRGRIPVNTRFQTKIPNIYAIGDVVAGPMLAHKAEDEGIICVEGMAGGAVHIDYNCVPSVIYTHPEVAWVGKSEEQLKEEGIEYKVGKFPFAANSRAKTNADTDGMVKILGQKSTDRVLGAHILGPGAGEMVNEAALALEYGASCEDIARVCHAHPTLSEAFREANLAASFGKSINF",
# ENOA_HUMAN
"MSILKIHAREIFDSRGNPTVEVDLFTSKGLFRAAVPSGASTGIYEALELRDNDKTRYMGKGVSKAVEHINKTIAPALVSKKLNVTEQEKIDKLMIEMDGTENKSKFGANAILGVSLAVCKAGAVEKGVPLYRHIADLAGNSEVILPVPAFNVINGGSHAGNKLAMQEFMILPVGAANFREAMRIGAEVYHNLKNVIKEKYGKDATNVGDEGGFAPNILENKEGLELLKTAIGKAGYTDKVVIGMDVAASEFFRSGKYDLDFKSPDDPSRYISPDQLADLYKSFIKDYPVVSIEDPFDQDDWGAWQKFTASAGIQVVGDDLTVTNPKRIAKAVNEKSCNCLLLKVNQIGSVTESLQACKLAQANGWGVMVSHRSGETEDTFIADLVVGLCTGQIKTGAPCRSERLAKYNQLLRIEEELGSKAKFAGRNFRNPLAK",
# ENOG_HUMAN
"MSIEKIWAREILDSRGNPTVEVDLYTAKGLFRAAVPSGASTGIYEALELRDGDKQRYLGKGVLKAVDHINSTIAPALISSGLSVVEQEKLDNLMLELDGTENKSKFGANAILGVSLAVCKAGAAERELPLYRHIAQLAGNSDLILPVPAFNVINGGSHAGNKLAMQEFMILPVGAESFRDAMRLGAEVYHTLKGVIKDKYGKDATNVGDEGGFAPNILENSEALELVKEAIDKAGYTEKIVIGMDVAASEFYRDGKYDLDFKSPTDPSRYITGDQLGALYQDFVRDYPVVSIEDPFDQDDWAAWSKFTANVGIQIVGDDLTVTNPKRIERAVEEKACNCLLLKVNQIGSVTEAIQACKLAQENGWGVMVSHRSGETEDTFIADLVVGLCTGQIKTGAPCRSERLAKYNQLMRIEEELGDEARFAGHNFRNPSVL",
# ENOB_HUMAN
"MAMQKIFAREILDSRGNPTVEVDLHTAKGRFRAAVPSGASTGIYEALELRDGDKGRYLGKGVLKAVENINNTLGPALLQKKLSVVDQEKVDKFMIELDGTENKSKFGANAILGVSLAVCKAGAAEKGVPLYRHIADLAGNPDLILPVPAFNVINGGSHAGNKLAMQEFMILPVGASSFKEAMRIGAEVYHHLKGVIKAKYGKDATNVGDEGGFAPNILENNEALELLKTAIQAAGYPDKVVIGMDVAASEFYRNGKYDLDFKSPDDPARHITGEKLGELYKSFIKNYPVVSIEDPFDQDDWATWTSFLSGVNIQIVGDDLTVTNPKRIAQAVEKKACNCLLLKVNQIGSVTESIQACKLAQSNGWGVMVSHRSGETEDTFIADLVVGLCTGQIKTGAPCRSERLAKYNQLMRIEEALGDKAIFAGRKFRNPKAK",
# F16P1_HUMAN
"MADQAPFDTDVNTLTRFVMEEGRKARGTGELTQLLNSLCTAVKAISSAVRKAGIAHLYGIAGSTNVTGDQVKKLDVLSNDLVMNMLKSSFATCVLVSEEDKHAIIVEPEKRGKYVVCFDPLDGSSNIDCLVSVGTIFGIYRKKSTDEPSEKDALQPGRNLVAAGYALYGSATMLVLAMDCGVNCFMLDPAIGEFILVDKDVKIKKKGKIYSLNEGYARDFDPAVTEYIQRKKFPPDNSAPYGARYVGSMVADVHRTLVYGGIFLYPANKKSPNGKLRLLYECNPMAYVMEKAGGMATTGKEAVLDVIPTDIHQRAPVILGSPDDVLEFLKVYEKHSAQ",
# F16P2_HUMAN
"MTDRSPFETDMLTLTRYVMEKGRQAKGTGELTQLLNSMLTAIKAISSAVRKAGLAHLYGIAGSVNVTGDEVKKLDVLSNSLVINMVQSSYSTCVLVSEENKDAIITAKEKRGKYVVCFDPLDGSSNIDCLASIGTIFAIYRKTSEDEPSEKDALQCGRNIVAAGYALYGSATLVALSTGQGVDLFMLDPALGEFVLVEKDVKIKKKGKIYSLNEGYAKYFDAATTEYVQKKKFPEDGSAPYGARYVGSMVADVHRTLVYGGIFLYPANQKSPKGKLRLLYECNPVAYIIEQAGGLATTGTQPVLDVKPEAIHQRVPLILGSPEDVQEYLTCVQKNQAGS",
# PCX4_HUMAN
"MSPDVPLLNDYKQDFFLKRFPQTVLGGPRFKLGYCAPPYIYVNQIILFLMPWVWGGVGTLLYQLGILKDYYTAALSGGLMLFTAFVIQFTSLYAKNKSTTVERILTTDILAEEDEHEFTSCTGAETVKFLIPGKKYVANTVFHSILAGLACGLGTWYLLPNRITLLYGSTGGTALLFFFGWMTLCIAEYSLIVNTATETATFQTQDTYEIIPLMRPLYIFFFVSVDLAHRFVVNMPALEHMNQILHILFVFLPFLWALGTLPPPDALLLWAMEQVLEFGLGGSSMSTHLRLLVMFIMSAGTAIASYFIPSTVGVVLFMTGFGFLLSLNLSDMGHKIGTKSKDLPSGPEKHFSWKECLFYIIILVLALLETSLLHHFAGFSQISKSNSQAIVGYGLMILLIILWILREIQSVYIIGIFRNPFYPKDVQTVTVFFEKQTRLMKIGIVRRILLTLVSPFAMIAFLSLDSSLQGLHSVSVCIGFTRAFRMVWQNTENALLETVIVSTVHLISSTDIWWNRSLDTGLRLLLVGIIRDRLIQFISKLQFAVTVLLTSWTEKKQRRKTTATLCILNIVFSPFVLVIIVFSTLLSSPLLPLFTLPVFLVGFPRPIQSWPGAAGTTACVCADTVYYYQMVPRLTAVLQTAMAAGSLGLLLPGSHYLGRFQDRLMWIMILECGYTYCSINIKGLELQETSCHTAEARRVDEVFEDAFEQEYTRVCSLNEHFGNVLTPCTVLPVKLYSDARNVLSGIIDSHENLKEFKGDLIKVLVWILVQYCSKRPGMKENVHNTENKGKAPLMLPALNTLPPPKSPEDIDSLNSETFNDWSDDNIFDDEPTIKKVIEEKHQLKDLPGTNLFIPGSVESQRVGDHSTGTVPENDLYKAVLLGYPAVDKGKQEDMPYIPLMEFSCSHSHLVCLPAEWRTSCMPSSKMKEMSSLFPEDWYQFVLRQLECYHSEEKASNVLEEIAKDKVLKDFYVHTVMTCYFSLFGIDNMAPSPGHILRVYGGVLPWSVALDWLTEKPELFQLALKAFRYTLKLMIDKASLGPIEDFRELIKYLEEYERDWYIGLVSDEKWKEAILQEKPYLFSLGYDSNMGIYTGRVLSLQELLIQVGKLNPEAVRGQWANLSWELLYATNDDEERYSIQAHPLLLRNLTVQAAEPPLGYPIYSSKPLHIHLY",
# G6PC1_HUMAN
"MEEGMNVLHDFGIQSTHYLQVNYQDSQDWFILVSVIADLRNAFYVLFPIWFHLQEAVGIKLLWVAVIGDWLNLVFKWILFGQRPYWWVLDTDYYSNTSVPLIKQFPVTCETGPGSPSGHAMGTAGVYYVMVTSTLSIFQGKIKPTYRFRCLNVILWLGFWAVQLNVCLSRIYLAAHFPHQVVAGVLSGIAVAETFSHIHSIYNASLKKYFLITFFLFSFAIGFYLLLKGLGVDLLWTLEKAQRWCEQPEWVHIDTTPFASLLKNLGTLFGLGLALNSSMYRESCKGKLSKWLPFRLSSIVASLVLLHVFDSLKPPSQVELVFYVLSFCKSAVVPLASVSVIPYCLAQVLGQPHKKSL",
# G3P_HUMAN
"MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHGKFHGTVKAENGKLVINGNPITIFQERDPSKIKWGDAGAEYVVESTGVFTTMEKAGAHLQGGAKRVIISAPSADAPMFVMGVNHEKYDNSLKIISNASCTTNCLAPLAKVIHDNFGIVEGLMTTVHAITATQKTVDGPSGKLWRDGRGALQNIIPASTGAAKAVGKVIPELNGKLTGMAFRVPTANVSVVDLTCRLEKPAKYDDIKKVVKQASEGPLKGILGYTEHQVVSSDFNSDTHSSTFDAGAGIALNDHFVKLISWYDNEFGYSNRVVDLMAHMASKE",
# HXK4_HUMAN
"MLDDRARMEAAKKEKVEQILAEFQLQEEDLKKVMRRMQKEMDRGLRLETHEEASVKMLPTYVRSTPEGSEVGDFLSLDLGGTNFRVMLVKVGEGEEGQWSVKTKHQMYSIPEDAMTGTAEMLFDYISECISDFLDKHQMKHKKLPLGFTFSFPVRHEDIDKGILLNWTKGFKASGAEGNNVVGLLRDAIKRRGDFEMDVVAMVNDTVATMISCYYEDHQCEVGMIVGTGCNACYMEEMQNVELVEGDEGRMCVNTEWGAFGDSGELDEFLLEYDRLVDESSANPGQQLYEKLIGGKYMGELVRLVLLRLVDENLLFHGEASEQLRTRGAFETRFVSQVESDTGDRKQIYNILSTLGLRPSTTDCDIVRRACESVSTRAAHMCSAGLAGVINRMRESRSEDVMRITVGVDGSVYKLHPSFKERFHASVRRLTPSCEITFIESEEGSGRGAALVSAVACKKACMLGQ",
# M4K2_HUMAN
"MALLRDVSLQDPRDRFELLQRVGAGTYGDVYKARDTVTSELAAVKIVKLDPGDDISSLQQEITILRECRHPNVVAYIGSYLRNDRLWICMEFCGGGSLQEIYHATGPLEERQIAYVCREALKGLHHLHSQGKIHRDIKGANLLLTLQGDVKLADFGVSGELTASVAKRRSFIGTPYWMAPEVAAVERKGGYNELCDVWALGITAIELGELQPPLFHLHPMRALMLMSKSSFQPPKLRDKTRWTQNFHHFLKLALTKNPKKRPTAEKLLQHPFTTQQLPRALLTQLLDKASDPHLGTPSPEDCELETYDMFPDTIHSRGQHGPAERTPSEIQFHQVKFGAPRRKETDPLNEPWEEEWTLLGKEELSGSLLQSVQEALEERSLTIRSASEFQELDSPDDTMGTIKRAPFLGPLPTDPPAEEPLSSPPGTLPPPPSGPNSSPLLPTAWATMKQREDPERSSCHGLPPTPKVHMGACFSKVFNGCPLRIHAAVTWIHPVTRDQFLVVGAEEGIYTLNLHELHEDTLEKLISHRCSWLYCVNNVLLSLSGKSTHIWAHDLPGLFEQRRLQQQVPLSIPTNRLTQRIIPRRFALSTKIPDTKGCLQCRVVRNPYTGATFLLAALPTSLLLLQWYEPLQKFLLLKNFSSPLPSPAGMLEPLVLDGKELPQVCVGAEGPEGPGCRVLFHVLPLEAGLTPDILIPPEGIPGSAQQVIQVDRDTILVSFERCVRIVNMQGEPTATLAPELTFDFPIETVVCLQDSVLAFWSHGMQGRSLDTNEVTQEITDETRIFRVLGAHRDIILESIPTDNPEAHSNLYILTGHQSTY",
# G6PI_HUMAN
"MAALTRDPQFQKLQQWYREHRSELNLRRLFDANKDRFNHFSLTLNTNHGHILVDYSKNLVTEDVMRMLVDLAKSRGVEAARERMFNGEKINYTEGRAVLHVALRNRSNTPILVDGKDVMPEVNKVLDKMKSFCQRVRSGDWKGYTGKTITDVINIGIGGSDLGPLMVTEALKPYSSGGPRVWYVSNIDGTHIAKTLAQLNPESSLFIIASKTFTTQETITNAETAKEWFLQAAKDPSAVAKHFVALSTNTTKVKEFGIDPQNMFEFWDWVGGRYSLWSAIGLSIALHVGFDNFEQLLSGAHWMDQHFRTTPLEKNAPVLLALLGIWYINCFGCETHAMLPYDQYLHRFAAYFQQGDMESNGKYITKSGTRVDHQTGPIVWGEPGTNGQHAFYQLIHQGTKMIPCDFLIPVQTQHPIRKGLHHKILLANFLAQTEALMRGKSTEEARKELQAAGKSPEDLERLLPHKVFEGNRPTNSIVFTKLTPFMLGALVAMYEHKIFVQGIIWDINSFDQWGVELGKQLAKKIEPELDGSAQVTSHDASTNGLINFIKQQREARVQ",
# HXK1_HUMAN
"MIAAQLLAYYFTELKDDQVKKIDKYLYAMRLSDETLIDIMTRFRKEMKNGLSRDFNPTATVKMLPTFVRSIPDGSEKGDFIALDLGGSSFRILRVQVNHEKNQNVHMESEVYDTPENIVHGSGSQLFDHVAECLGDFMEKRKIKDKKLPVGFTFSFPCQQSKIDEAILITWTKRFKASGVEGADVVKLLNKAIKKRGDYDANIVAVVNDTVGTMMTCGYDDQHCEVGLIIGTGTNACYMEELRHIDLVEGDEGRMCINTEWGAFGDDGSLEDIRTEFDREIDRGSLNPGKQLFEKMVSGMYLGELVRLILVKMAKEGLLFEGRITPELLTRGKFNTSDVSAIEKNKEGLHNAKEILTRLGVEPSDDDCVSVQHVCTIVSFRSANLVAATLGAILNRLRDNKGTPRLRTTVGVDGSLYKTHPQYSRRFHKTLRRLVPDSDVRFLLSESGSGKGAAMVTAVAYRLAEQHRQIEETLAHFHLTKDMLLEVKKRMRAEMELGLRKQTHNNAVVKMLPSFVRRTPDGTENGDFLALDLGGTNFRVLLVKIRSGKKRTVEMHNKIYAIPIEIMQGTGEELFDHIVSCISDFLDYMGIKGPRMPLGFTFSFPCQQTSLDAGILITWTKGFKATDCVGHDVVTLLRDAIKRREEFDLDVVAVVNDTVGTMMTCAYEEPTCEVGLIVGTGSNACYMEEMKNVEMVEGDQGQMCINMEWGAFGDNGCLDDIRTHYDRLVDEYSLNAGKQRYEKMISGMYLGEIVRNILIDFTKKGFLFRGQISETLKTRGIFETKFLSQIESDRLALLQVRAILQQLGLNSTCDDSILVKTVCGVVSRRAAQLCGAGMAAVVDKIRENRGLDRLNVTVGVDGTLYKLHPHFSRIMHQTVKELSPKCNVSFLLSEDGSGKGAALITAVGVRLRTEASS",
# HXK2_HUMAN
"MIASHLLAYFFTELNHDQVQKVDQYLYHMRLSDETLLEISKRFRKEMEKGLGATTHPTAAVKMLPTFVRSTPDGTEHGEFLALDLGGTNFRVLWVKVTDNGLQKVEMENQIYAIPEDIMRGSGTQLFDHIAECLANFMDKLQIKDKKLPLGFTFSFPCHQTKLDESFLVSWTKGFKSSGVEGRDVVALIRKAIQRRGDFDIDIVAVVNDTVGTMMTCGYDDHNCEIGLIVGTGSNACYMEEMRHIDMVEGDEGRMCINMEWGAFGDDGSLNDIRTEFDQEIDMGSLNPGKQLFEKMISGMYMGELVRLILVKMAKEELLFGGKLSPELLNTGRFETKDISDIEGEKDGIRKAREVLMRLGLDPTQEDCVATHRICQIVSTRSASLCAATLAAVLQRIKENKGEERLRSTIGVDGSVYKKHPHFAKRLHKTVRRLVPGCDVRFLRSEDGSGKGAAMVTAVAYRLADQHRARQKTLEHLQLSHDQLLEVKRRMKVEMERGLSKETHASAPVKMLPTYVCATPDGTEKGDFLALDLGGTNFRVLLVRVRNGKWGGVEMHNKIYAIPQEVMHGTGDELFDHIVQCIADFLEYMGMKGVSLPLGFTFSFPCQQNSLDESILLKWTKGFKASGCEGEDVVTLLKEAIHRREEFDLDVVAVVNDTVGTMMTCGFEDPHCEVGLIVGTGSNACYMEEMRNVELVEGEEGRMCVNMEWGAFGDNGCLDDFRTEFDVAVDELSLNPGKQRFEKMISGMYLGEIVRNILIDFTKRGLLFRGRISERLKTRGIFETKFLSQIESDCLALLQVRAILQHLGLESTCDDSIIVKEVCTVVARRAAQLCGAGMAAVVDRIRENRGLDALKVTVGVDGTLYKLHPHFAKVMHETVKDLAPKCDVSFLQSEDGSGKGAALITAVACRIREAGQR",
# HKDC1_HUMAN
"MFAVHLMAFYFSKLKEDQIKKVDRFLYHMRLSDDTLLDIMRRFRAEMEKGLAKDTNPTAAVKMLPTFVRAIPDGSENGEFLSLDLGGSKFRVLKVQVAEEGKRHVQMESQFYPTPNEIIRGNGTELFEYVADCLADFMKTKDLKHKKLPLGLTFSFPCRQTKLEEGVLLSWTKKFKARGVQDTDVVSRLTKAMRRHKDMDVDILALVNDTVGTMMTCAYDDPYCEVGVIIGTGTNACYMEDMSNIDLVEGDEGRMCINTEWGAFGDDGALEDIRTEFDRELDLGSLNPGKQLFEKMISGLYLGELVRLILLKMAKAGLLFGGEKSSALHTKGKIETRHVAAMEKYKEGLANTREILVDLGLEPSEADCIAVQHVCTIVSFRSANLCAAALAAILTRLRENKKVERLRTTVGMDGTLYKIHPQYPKRLHKVVRKLVPSCDVRFLLSESGSTKGAAMVTAVASRVQAQRKQIDRVLALFQLTREQLVDVQAKMRAELEYGLKKKSHGLATVRMLPTYVCGLPDGTEKGKFLALDLGGTNFRVLLVKIRSGRRSVRMYNKIFAIPLEIMQGTGEELFDHIVQCIADFLDYMGLKGASLPLGFTFSFPCRQMSIDKGTLIGWTKGFKATDCEGEDVVDMLREAIKRRNEFDLDIVAVVNDTVGTMMTCGYEDPNCEIGLIAGTGSNMCYMEDMRNIEMVEGGEGKMCINTEWGGFGDNGCIDDIWTRYDTEVDEGSLNPGKQRYEKMTSGMYLGEIVRQILIDLTKQGLLFRGQISERLRTRGIFETKFLSQIESDRLALLQVRRILQQLGLDSTCEDSIVVKEVCGAVSRRAAQLCGAGLAAIVEKRREDQGLEHLRITVGVDGTLYKLHPHFSRILQETVKELAPRCDVTFMLSEDGSGKGAALITAVAKRLQQAQKEN",
# LDHA_HUMAN
"MATLKDQLIYNLLKEEQTPQNKITVVGVGAVGMACAISILMKDLADELALVDVIEDKLKGEMMDLQHGSLFLRTPKIVSGKDYNVTANSKLVIITAGARQQEGESRLNLVQRNVNIFKFIIPNVVKYSPNCKLLIVSNPVDILTYVAWKISGFPKNRVIGSGCNLDSARFRYLMGERLGVHPLSCHGWVLGEHGDSSVPVWSGMNVAGVSLKTLHPDLGTDKDKEQWKEVHKQVVESAYEVIKLKGYTSWAIGLSVADLAESIMKNLRRVHPVSTMIKGLYGIKDDVFLSVPCILGQNGISDLVKVTLTSEEEARLKKSADTLWGIQKELQF",
# LDHB_HUMAN
"MATLKEKLIAPVAEEEATVPNNKITVVGVGQVGMACAISILGKSLADELALVDVLEDKLKGEMMDLQHGSLFLQTPKIVADKDYSVTANSKIVVVTAGVRQQEGESRLNLVQRNVNVFKFIIPQIVKYSPDCIIIVVSNPVDILTYVTWKLSGLPKHRVIGSGCNLDSARFRYLMAEKLGIHPSSCHGWILGEHGDSSVAVWSGVNVAGVSLQELNPEMGTDNDSENWKEVHKMVVESAYEVIKLKGYTNWAIGLSVADLIESMLKNLSRIHPVSTMVKGMYGIENEVFLSLPCILNARGLTSVINQKLKDDEVAQLKKSADTLWDIQKDLKDL",
# LDHC_HUMAN
"MSTVKEQLIEKLIEDDENSQCKITIVGTGAVGMACAISILLKDLADELALVDVALDKLKGEMMDLQHGSLFFSTSKITSGKDYSVSANSRIVIVTAGARQQEGETRLALVQRNVAIMKSIIPAIVHYSPDCKILVVSNPVDILTYIVWKISGLPVTRVIGSGCNLDSARFRYLIGEKLGVHPTSCHGWIIGEHGDSSVPLWSGVNVAGVALKTLDPKLGTDSDKEHWKNIHKQVIQSAYEIIKLKGYTSWAIGLSVMDLVGSILKNLRRVHPVSTMVKGLYGIKEELFLSIPCVLGRNGVSDVVKINLNSEEEALFKKSAETLWNIQKDLIF",
# ODPA_HUMAN
"MRKMLAAVSRVLSGASQKPASRVLVASRNFANDATFEIKKCDLHRLEEGPPVTTVLTREDGLKYYRMMQTVRRMELKADQLYKQKIIRGFCHLCDGQEACCVGLEAGINPTDHLITAYRAHGFTFTRGLSVREILAELTGRKGGCAKGKGGSMHMYAKNFYGGNGIVGAQVPLGAGIALACKYNGKDEVCLTLYGDGAANQGQIFEAYNMAALWKLPCIFICENNRYGMGTSVERAAASTDYYKRGDFIPGLRVDGMDILCVREATRFAAAYCRSGKGPILMELQTYRYHGHSMSDPGVSYRTREEIQEVRSKSDPIMLLKDRMVNSNLASVEELKEIDVEVRKEIEDAAQFATADPEPPLEELGYHIYSSDPPFEVRGANQWIKFKSVS",
# ODPAT_HUMAN
"MLAAFISRVLRRVAQKSARRVLVASRNSSNDATFEIKKCDLYLLEEGPPVTTVLTRAEGLKYYRMMLTVRRMELKADQLYKQKFIRGFCHLCDGQEACCVGLEAGINPSDHVITSYRAHGVCYTRGLSVRSILAELTGRRGGCAKGKGGSMHMYTKNFYGGNGIVGAQGPLGAGIALACKYKGNDEICLTLYGDGAANQGQIAEAFNMAALWKLPCVFICENNLYGMGTSTERAAASPDYYKRGNFIPGLKVDGMDVLCVREATKFAANYCRSGKGPILMELQTYRYHGHSMSDPGVSYRTREEIQEVRSKRDPIIILQDRMVNSKLATVEELKEIGAEVRKEIDDAAQFATTDPEPHLEELGHHIYSSDSSFEVRGANPWIKFKSVS",
# ODPB_HUMAN
"MAAVSGLVRRPLREVSGLLKRRFHWTAPAALQVTVRDAINQGMDEELERDEKVFLLGEEVAQYDGAYKVSRGLWKKYGDKRIIDTPISEMGFAGIAVGAAMAGLRPICEFMTFNFSMQAIDQVINSAAKTYYMSGGLQPVPIVFRGPNGASAGVAAQHSQCFAAWYGHCPGLKVVSPWNSEDAKGLIKSAIRDNNPVVVLENELMYGVPFEFPPEAQSKDFLIPIGKAKIERQGTHITVVSHSRPVGHCLEAAAVLSKEGVECEVINMRTIRPMDMETIEASVMKTNHLVTVEGGWPQFGVGAEICARIMEGPAFNFLDAPAVRVTGADVPMPYAKILEDNSIPQVKDIIFAIKKTLNI",
# PFKAL_HUMAN
"MAAVDLEKLRASGAGKAIGVLTSGGDAQGMNAAVRAVTRMGIYVGAKVFLIYEGYEGLVEGGENIKQANWLSVSNIIQLGGTIIGSARCKAFTTREGRRAAAYNLVQHGITNLCVIGGDGSLTGANIFRSEWGSLLEELVAEGKISETTARTYSHLNIAGLVGSIDNDFCGTDMTIGTDSALHRIMEVIDAITTTAQSHQRTFVLEVMGRHCGYLALVSALASGADWLFIPEAPPEDGWENFMCERLGETRSRGSRLNIIIIAEGAIDRNGKPISSSYVKDLVVQRLGFDTRVTVLGHVQRGGTPSAFDRILSSKMGMEAVMALLEATPDTPACVVTLSGNQSVRLPLMECVQMTKEVQKAMDDKRFDEATQLRGGSFENNWNIYKLLAHQKPPKEKSNFSLAILNVGAPAAGMNAAVRSAVRTGISHGHTVYVVHDGFEGLAKGQVQEVGWHDVAGWLGRGGSMLGTKRTLPKGQLESIVENIRIYGIHALLVVGGFEAYEGVLQLVEARGRYEELCIVMCVIPATISNNVPGTDFSLGSDTAVNAAMESCDRIKQSASGTKRRVFIVETMGGYCGYLATVTGIAVGADAAYVFEDPFNIHDLKVNVEHMTEKMKTDIQRGLVLRNEKCHDYYTTEFLYNLYSSEGKGVFDCRTNVLGHLQQGGAPTPFDRNYGTKLGVKAMLWLSEKLREVYRKGRVFANAPDSACVIGLKKKAVAFSPVTELKKDTDFEHRMPREQWWLSLRLMLKMLAQYRISMAAYVSGELEHVTRRTLSMDKGF",
# PFKAM_HUMAN
"MTHEEHHAAKTLGIGKAIAVLTSGGDAQGMNAAVRAVVRVGIFTGARVFFVHEGYQGLVDGGDHIKEATWESVSMMLQLGGTVIGSARCKDFREREGRLRAAYNLVKRGITNLCVIGGDGSLTGADTFRSEWSDLLSDLQKAGKITDEEATKSSYLNIVGLVGSIDNDFCGTDMTIGTDSALHRIMEIVDAITTTAQSHQRTFVLEVMGRHCGYLALVTSLSCGADWVFIPECPPDDDWEEHLCRRLSETRTRGSRLNIIIVAEGAIDKNGKPITSEDIKNLVVKRLGYDTRVTVLGHVQRGGTPSAFDRILGSRMGVEAVMALLEGTPDTPACVVSLSGNQAVRLPLMECVQVTKDVTKAMDEKKFDEALKLRGRSFMNNWEVYKLLAHVRPPVSKSGSHTVAVMNVGAPAAGMNAAVRSTVRIGLIQGNRVLVVHDGFEGLAKGQIEEAGWSYVGGWTGQGGSKLGTKRTLPKKSFEQISANITKFNIQGLVIIGGFEAYTGGLELMEGRKQFDELCIPFVVIPATVSNNVPGSDFSVGADTALNTICTTCDRIKQSAAGTKRRVFIIETMGGYCGYLATMAGLAAGADAAYIFEEPFTIRDLQANVEHLVQKMKTTVKRGLVLRNEKCNENYTTDFIFNLYSEEGKGIFDSRKNVLGHMQQGGSPTPFDRNFATKMGAKAMNWMSGKIKESYRNGRIFANTPDSGCVLGMRKRALVFQPVAELKDQTDFEHRIPKEQWWLKLRPILKILAKYEIDLDTSDHAHLEHITRKRSGEAAV",
# PFKAP_HUMAN
"MDADDSRAPKGSLRKFLEHLSGAGKAIGVLTSGGDAQGMNAAVRAVVRMGIYVGAKVYFIYEGYQGMVDGGSNIAEADWESVSSILQVGGTIIGSARCQAFRTREGRLKAACNLLQRGITNLCVIGGDGSLTGANLFRKEWSGLLEELARNGQIDKEAVQKYAYLNVVGMVGSIDNDFCGTDMTIGTDSALHRIIEVVDAIMTTAQSHQRTFVLEVMGRHCGYLALVSALACGADWVFLPESPPEEGWEEQMCVKLSENRARKKRLNIIIVAEGAIDTQNKPITSEKIKELVVTQLGYDTRVTILGHVQRGGTPSAFDRILASRMGVEAVIALLEATPDTPACVVSLNGNHAVRLPLMECVQMTQDVQKAMDERRFQDAVRLRGRSFAGNLNTYKRLAIKLPDDQIPKTNCNVAVINVGAPAAGMNAAVRSAVRVGIADGHRMLAIYDGFDGFAKGQIKEIGWTDVGGWTGQGGSILGTKRVLPGKYLEEIATQMRTHSINALLIIGGFEAYLGLLELSAAREKHEEFCVPMVMVPATVSNNVPGSDFSIGADTALNTITDTCDRIKQSASGTKRRVFIIETMGGYCGYLANMGGLAAGADAAYIFEEPFDIRDLQSNVEHLTEKMKTTIQRGLVLRNESCSENYTTDFIYQLYSEEGKGVFDCRKNVLGHMQQGGAPSPFDRNFGTKISARAMEWITAKLKEARGRGKKFTTDDSICVLGISKRNVIFQPVAELKKQTDFEHRIPKEQWWLKLRPLMKILAKYKASYDVSDSGQLEHVQPWSV",
# PGAM1_HUMAN
"MAAYKLVLIRHGESAWNLENRFSGWYDADLSPAGHEEAKRGGQALRDAGYEFDICFTSVQKRAIRTLWTVLDAIDQMWLPVVRTWRLNERHYGGLTGLNKAETAAKHGEAQVKIWRRSYDVPPPPMEPDHPFYSNISKDRRYADLTEDQLPSCESLKDTIARALPFWNEEIVPQIKEGKRVLIAAHGNSLRGIVKHLEGLSEEAIMELNLPTGIPIVYELDKNLKPIKPMQFLGDEETVRKAMEAVAAQGKAKK",
# PGAM2_HUMAN
"MATHRLVMVRHGESTWNQENRFCGWFDAELSEKGTEEAKRGAKAIKDAKMEFDICYTSVLKRAIRTLWAILDGTDQMWLPVVRTWRLNERHYGGLTGLNKAETAAKHGEEQVKIWRRSFDIPPPPMDEKHPYYNSISKERRYAGLKPGELPTCESLKDTIARALPFWNEEIVPQIKAGKRVLIAAHGNSLRGIVKHLEGMSDQAIMELNLPTGIPIVYELNKELKPTKPMQFLGDEETVRKAMEAVAAQGKAK",
# PGAM4_HUMAN
"MAAYKLVLIRHGESTWNLENRFSCWYDADLSPAGHEEAKRGGQALRDAGYEFDICLTSVQKRVIRTLWTVLDAIDQMWLPVVRTWRLNERHYGGLTGLNKAETAAKHGEAQVKIWRRSYDVPPPPMEPDHPFYSNISKDRRYADLTEDQLPSYESPKDTIARALPFWNEEIVPQIKEGKRVLIAAHGNSLQGIAKHVEGLSEEAIMELNLPTGIPIVYELDKNLKPIKPMQFLGDEETVCKAIEAVAAQGKAKK",
# PGK1_HUMAN
"MSLSNKLTLDKLDVKGKRVVMRVDFNVPMKNNQITNNQRIKAAVPSIKFCLDNGAKSVVLMSHLGRPDGVPMPDKYSLEPVAVELKSLLGKDVLFLKDCVGPEVEKACANPAAGSVILLENLRFHVEEEGKGKDASGNKVKAEPAKIEAFRASLSKLGDVYVNDAFGTAHRAHSSMVGVNLPQKAGGFLMKKELNYFAKALESPERPFLAILGGAKVADKIQLINNMLDKVNEMIIGGGMAFTFLKVLNNMEIGTSLFDEEGAKIVKDLMSKAEKNGVKITLPVDFVTADKFDENAKTGQATVASGIPAGWMGLDCGPESSKKYAEAVTRAKQIVWNGPVGVFEWEAFARGTKALMDEVVKATSRGCITIIGGGDTATCCAKWNTEDKVSHVSTGGGASLELLEGKVLPGVDALSNI",
# PGM1_HUMAN
"MVKIVTVKTQAYQDQKPGTSGLRKRVKVFQSSANYAENFIQSIISTVEPAQRQEATLVVGGDGRFYMKEAIQLIARIAAANGIGRLVIGQNGILSTPAVSCIIRKIKAIGGIILTASHNPGGPNGDFGIKFNISNGGPAPEAITDKIFQISKTIEEYAVCPDLKVDLGVLGKQQFDLENKFKPFTVEIVDSVEAYATMLRSIFDFSALKELLSGPNRLKIRIDAMHGVVGPYVKKILCEELGAPANSAVNCVPLEDFGGHHPDPNLTYAADLVETMKSGEHDFGAAFDGDGDRNMILGKHGFFVNPSDSVAVIAANIFSIPYFQQTGVRGFARSMPTSGALDRVASATKIALYETPTGWKFFGNLMDASKLSLCGEESFGTGSDHIREKDGLWAVLAWLSILATRKQSVEDILKDHWQKYGRNFFTRYDYEEVEAEGANKMMKDLEALMFDRSFVGKQFSANDKVYTVEKADNFEYSDPVDGSISRNQGLRLIFTDGSRIVFRLSGTGSAGATIRLYIDSYEKDVAKINQDPQVMLAPLISIALKVSQLQERTGRTAPTVIT",
# KPYR_HUMAN
"MSIQENISSLQLRSWVSKSQRDLAKSILIGAPGGPAGYLRRASVAQLTQELGTAFFQQQQLPAAMADTFLEHLCLLDIDSEPVAARSTSIIATIGPASRSVERLKEMIKAGMNIARLNFSHGSHEYHAESIANVREAVESFAGSPLSYRPVAIALDTKGPEIRTGILQGGPESEVELVKGSQVLVTVDPAFRTRGNANTVWVDYPNIVRVVPVGGRIYIDDGLISLVVQKIGPEGLVTQVENGGVLGSRKGVNLPGAQVDLPGLSEQDVRDLRFGVEHGVDIVFASFVRKASDVAAVRAALGPEGHGIKIISKIENHEGVKRFDEILEVSDGIMVARGDLGIEIPAEKVFLAQKMMIGRCNLAGKPVVCATQMLESMITKPRPTRAETSDVANAVLDGADCIMLSGETAKGNFPVEAVKMQHAIAREAEAAVYHRQLFEELRRAAPLSRDPTEVTAIGAVEAAFKCCAAAIIVLTTTGRSAQLLSRYRPRAAVIAVTRSAQAARQVHLCRGVFPLLYREPPEAIWADDVDRRVQFGIESGKLRGFLRVGDLVIVVTGWRPGSGYTNIMRVLSIS",
# KPYM_HUMAN
"MSKPHSEAGTAFIQTQQLHAAMADTFLEHMCRLDIDSPPITARNTGIICTIGPASRSVETLKEMIKSGMNVARLNFSHGTHEYHAETIKNVRTATESFASDPILYRPVAVALDTKGPEIRTGLIKGSGTAEVELKKGATLKITLDNAYMEKCDENILWLDYKNICKVVEVGSKIYVDDGLISLQVKQKGADFLVTEVENGGSLGSKKGVNLPGAAVDLPAVSEKDIQDLKFGVEQDVDMVFASFIRKASDVHEVRKVLGEKGKNIKIISKIENHEGVRRFDEILEASDGIMVARGDLGIEIPAEKVFLAQKMMIGRCNRAGKPVICATQMLESMIKKPRPTRAEGSDVANAVLDGADCIMLSGETAKGDYPLEAVRMQHLIAREAEAAIYHLQLFEELRRLAPITSDPTEATAVGAVEASFKCCSGAIIVLTKSGRSAHQVARYRPRAPIIAVTRNPQTARQAHLYRGIFPVLCKDPVQEAWAEDVDLRVNFAMNVGKARGFFKKGDVVIVLTGWRPGSGFTNTMRVVPVP",
# TPIS_HUMAN
"MAPSRKFFVGGNWKMNGRKQSLGELIGTLNAAKVPADTEVVCAPPTAYIDFARQKLDPKIAVAAQNCYKVTNGAFTGEISPGMIKDCGATWVVLGHSERRHVFGESDELIGQKVAHALAEGLGVIACIGEKLDEREAGITEKVVFEQTKVIADNVKDWSKVVLAYEPVWAIGTGKTATPQQAQEVHEKLRGWLKSNVSDAVAQSTRIIYGGSVTGATCKELASQPDVDGFLVGGASLKPEFVDIINAKQ"
]


# Add protein names corresponding to the sequences
protein_names = [
"ADH1A_HUMAN",
"ADH4_HUMAN",
"ADH6_HUMAN",
"ADH7_HUMAN",
"HOT_HUMAN",
"AK1A1_HUMAN",
"AL1A1_HUMAN",
"AL1A2_HUMAN",
"AL1B1_HUMAN",
"ALDH2_HUMAN",
"AL3A1_HUMAN",
"AL3A2_HUMAN",
"AL3B1_HUMAN",
"AL9A1_HUMAN",
"ALDOA_HUMAN",
"ALDOB_HUMAN",
"ALDOC_HUMAN",
"PMGE_HUMAN",
"ODP2_HUMAN",
"DLDH_HUMAN",
"ENOA_HUMAN",
"ENOG_HUMAN",
"ENOB_HUMAN",
"F16P1_HUMAN",
"F16P2_HUMAN",
"PCX4_HUMAN",
"G6PC1_HUMAN",
"G3P_HUMAN",
"HXK4_HUMAN",
"M4K2_HUMAN",
"G6PI_HUMAN",
"HXK1_HUMAN",
"HXK2_HUMAN",
"HKDC1_HUMAN",
"LDHA_HUMAN",
"LDHB_HUMAN",
"LDHC_HUMAN",
"ODPA_HUMAN",
"ODPAT_HUMAN",
"ODPB_HUMAN",
"PFKAL_HUMAN",
"PFKAM_HUMAN",
"PFKAP_HUMAN",
"PGAM1_HUMAN",
"PGAM2_HUMAN",
"PGAM4_HUMAN",
"PGK1_HUMAN",
"PGM1_HUMAN",
"KPYR_HUMAN",
"KPYM_HUMAN",
"TPIS_HUMAN"

]

In [31]:
#Phosphotase
sequences = [
# PPBI_HUMAN
"MQGPWVLLLLGLRLQLSLGVIPAEEENPAFWNRQAAEALDAAKKLQPIQKVAKNLILFLGDGLGVPTVTATRILKGQKNGKLGPETPLAMDRFPYLALSKTYNVDRQVPDSAATATAYLCGVKANFQTIGLSAAARFNQCNTTRGNEVISVMNRAKQAGKSVGVVTTTRVQHASPAGTYAHTVNRNWYSDADMPASARQEGCQDIATQLISNMDIDVILGGGRKYMFPMGTPDPEYPADASQNGIRLDGKNLVQEWLAKHQGAWYVWNRTELMQASLDQSVTHLMGLFEPGDTKYEIHRDPTLDPSLMEMTEAALRLLSRNPRGFYLFVEGGRIDHGHHEGVAYQALTEAVMFDDAIERAGQLTSEEDTLTLVTADHSHVFSFGGYTLRGSSIFGLAPSKAQDSKAYTSILYGNGPGYVFNSGVRPDVNESESGSPDYQQQAAVPLSSETHGGEDVAVFARGPQAHLVHGVQEQSFVAHVMAFAACLEPYTACDLAPPACTTDAAHPVAASLPLLAGTLLLLGASAAP",
# PPBT_HUMAN
"MISPFLVLAIGTCLTNSLVPEKEKDPKYWRDQAQETLKYALELQKLNTNVAKNVIMFLGDGMGVSTVTAARILKGQLHHNPGEETRLEMDKFPFVALSKTYNTNAQVPDSAGTATAYLCGVKANEGTVGVSAATERSRCNTTQGNEVTSILRWAKDAGKSVGIVTTTRVNHATPSAAYAHSADRDWYSDNEMPPEALSQGCKDIAYQLMHNIRDIDVIMGGGRKYMYPKNKTDVEYESDEKARGTRLDGLDLVDTWKSFKPRYKHSHFIWNRTELLTLDPHNVDYLLGLFEPGDMQYELNRNNVTDPSLSEMVVVAIQILRKNPKGFFLLVEGGRIDHGHHEGKAKQALHEAVEMDRAIGQAGSLTSSEDTLTVVTADHSHVFTFGGYTPRGNSIFGLAPMLSDTDKKPFTAILYGNGPGYKVVGGERENVSMVDYAHNNYQAQSAVPLRHETHGGEDVAVFSKGPMAHLLHGVHEQNYVPHVMAYAACIGANLGHCAPASSAGSLAAGPLLLALALYPLSVLF",
# PPB1_HUMAN
"MLGPCMLLLLLLLGLRLQLSLGIIPVEEENPDFWNREAAEALGAAKKLQPAQTAAKNLIIFLGDGMGVSTVTAARILKGQKKDKLGPEIPLAMDRFPYVALSKTYNVDKHVPDSGATATAYLCGVKGNFQTIGLSAAARFNQCNTTRGNEVISVMNRAKKAGKSVGVVTTTRVQHASPAGTYAHTVNRNWYSDADVPASARQEGCQDIATQLISNMDIDVILGGGRKYMFRMGTPDPEYPDDYSQGGTRLDGKNLVQEWLAKRQGARYVWNRTELMQASLDPSVTHLMGLFEPGDMKYEIHRDSTLDPSLMEMTEAALRLLSRNPRGFFLFVEGGRIDHGHHESRAYRALTETIMFDDAIERAGQLTSEEDTLSLVTADHSHVFSFGGYPLRGSSIFGLAPGKARDRKAYTVLLYGNGPGYVLKDGARPDVTESESGSPEYRQQSAVPLDEETHAGEDVAVFARGPQAHLVHGVQEQTFIAHVMAFAACLEPYTACDLAPPAGTTDAAHPGRSVVPALLPLLAGTLLLLETATAP",
# PPBN_HUMAN
"MQGPWVLLLLGLRLQLSLGIIPVEEENPDFWNRQAAEALGAAKKLQPAQTAAKNLIIFLGDGMGVSTVTAARILKGQKKDKLGPETFLAMDRFPYVALSKTYSVDKHVPDSGATATAYLCGVKGNFQTIGLSAAARFNQCNTTRGNEVISVMNRAKKAGKSVGVVTTTRVQHASPAGAYAHTVNRNWYSDADVPASARQEGCQDIATQLISNMDIDVILGGGRKYMFPMGTPDPEYPDDYSQGGTRLDGKNLVQEWLAKHQGARYVWNRTELLQASLDPSVTHLMGLFEPGDMKYEIHRDSTLDPSLMEMTEAALLLLSRNPRGFFLFVEGGRIDHGHHESRAYRALTETIMFDDAIERAGQLTSEEDTLSLVTADHSHVFSFGGYPLRGSSIFGLAPGKARDRKAYTVLLYGNGPGYVLKDGARPDVTESESGSPEYRQQSAVPLDGETHAGEDVAVFARGPQAHLVHGVQEQTFIAHVMAFAACLEPYTACDLAPRAGTTDAAHPGPSVVPALLPLLAGTLLLLGTATAP",
# CC14A_HUMAN
"MAAESGELIGACEFMKDRLYFATLRNRPKSTVNTHYFSIDEELVYENFYADFGPLNLAMVYRYCCKLNKKLKSYSLSRKKIVHYTCFDQRKRANAAFLIGAYAVIYLKKTPEEAYRALLSGSNPPYLPFRDASFGNCTYNLTILDCLQGIRKGLQHGFFDFETFDVDEYEHYERVENGDFNWIVPGKFLAFSGPHPKSKIENGYPLHAPEAYFPYFKKHNVTAVVRLNKKIYEAKRFTDAGFEHYDLFFIDGSTPSDNIVRRFLNICENTEGAIAVHCKAGLGRTGTLIACYVMKHYRFTHAEIIAWIRICRPGSIIGPQQHFLEEKQASLWVQGDIFRSKLKNRPSSEGSINKILSGLDDMSIGGNLSKTQNMERFGEDNLEDDDVEMKNGITQGDKLRALKSQRQPRTSPSCAFRSDDTKGHPRAVSQPFRLSSSLQGSAVTLKTSKMALSPSATAKRINRTSLSSGATVRSFSINSRLASSLGNLNAATDDPENKKTSSSSKAGFTASPFTNLLNGSSQPTTRNYPELNNNQYNRSSNSNGGNLNSPPGPHSAKTEEHTTILRPSYTGLSSSSARFLSRSIPSLQSEYVHY",
# CC14B_HUMAN
"MKRKSERRSSWAAAPPCSRRCSSTSPGVKKIRSSTQQDPRRRDPQDDVYLDITDRLCFAILYSRPKSASNVHYFSIDNELEYENFYADFGPLNLAMVYRYCCKINKKLKSITMLRKKIVHFTGSDQRKQANAAFLVGCYMVIYLGRTPEEAYRILIFGETSYIPFRDAAYGSCNFYITLLDCFHAVKKAMQYGFLNFNSFNLDEYEHYEKAENGDLNWIIPDRFIAFCGPHSRARLESGYHQHSPETYIQYFKNHNVTTIIRLNKRMYDAKRFTDAGFDHHDLFFADGSTPTDAIVKEFLDICENAEGAIAVHCKAGLGRTGTLIACYIMKHYRMTAAETIAWVRICRPGSVIGPQQQFLVMKQTNLWLEGDYFRQKLKGQENGQHRAAFSKLLSGVDDISINGVENQDQQEPEPYSDDDEINGVTQGDRLRALKSRRQSKTNAIPLTVILQSSVQSCKTSEPNISGSAGITKRTTRSASRKSSVKSLSISRTKTVLR",
# CDK3_HUMAN
"MDMFQKVEKIGEGTYGVVYKAKNRETGQLVALKKIRLDLEMEGVPSTAIREISLLKELKHPNIVRLLDVVHNERKLYLVFEFLSQDLKKYMDSTPGSELPLHLIKSYLFQLLQGVSFCHSHRVIHRDLKPQNLLINELGAIKLADFGLARAFGVPLRTYTHEVVTLWYRAPEILLGSKFYTTAVDIWSIGCIFAEMVTRKALFPGDSEIDQLFRIFRMLGTPSEDTWPGVTQLPDYKGSFPKWTRKGLEEIVPNLEPEGRDLLMQLLQYDPSQRITAKTALAHPYFSSPEPSPAARQYVLQRFRH",
# CDKN3_HUMAN
"MKPPSSIQTSEFDSSDEEPIEDEQTPIHISWLSLSRVNCSQFLGLCALPGCKFKDVRRNVQKDTEELKSCGIQDIFVFCTRGELSKYRVPNLLDLYQQCGIITHHHPIADGGTPDIASCCEIMEELTTCLKNYRKTLIHCYGGLGRSCLVAACLLLYLSDTISPEQAIDSLRDLRGSGAIQTIKQYNYLHEFRDKLAAHLSSRDSQSRSVSR",
# DUS1_HUMAN
"MVMEVGTLDAGGLRALLGERAAQCLLLDCRSFFAFNAGHIAGSVNVRFSTIVRRRAKGAMGLEHIVPNAELRGRLLAGAYHAVVLLDERSAALDGAKRDGTLALAAGALCREARAAQVFFLKGGYEAFSASCPELCSKQSTPMGLSLPLSTSVPDSAESGCSSCSTPLYDQGGPVEILPFLYLGSAYHASRKDMLDALGITALINVSANCPNHFEGHYQYKSIPVEDNHKADISSWFNEAIDFIDSIKNAGGRVFVHCQAGISRSATICLAYLMRTNRVKLDEAFEFVKQRRSIISPNFSFMGQLLQFESQVLAPHCSAEAGSPAMAVLDRGTSTTTVFNFPVSIPVHSTNSALSYLQSPITTSPSC",
# DUSP2_HUMAN
"MGLEAARELECAALGTLLRDPREAERTLLLDCRPFLAFCRRHVRAARPVPWNALLRRRARGPPAAVLACLLPDRALRTRLVRGELARAVVLDEGSASVAELRPDSPAHVLLAALLHETRAGPTAVYFLRGGFDGFQGCCPDLCSEAPAPALPPTGDKTSRSDSRAPVYDQGGPVEILPYLFLGSCSHSSDLQGLQACGITAVLNVSASCPNHFEGLFRYKSIPVEDNQMVEISAWFQEAIGFIDWVKNSGGRVLVHCQAGISRSATICLAYLMQSRRVRLDEAFDFVKQRRGVISPNFSFMGQLLQFETQVLCH",
# DUS4_HUMAN
"MVTMEELREMDCSVLKRLMNRDENGGGAGGSGSHGTLGLPSGGKCLLLDCRPFLAHSAGYILGSVNVRCNTIVRRRAKGSVSLEQILPAEEEVRARLRSGLYSAVIVYDERSPRAESLREDSTVSLVVQALRRNAERTDICLLKGGYERFSSEYPEFCSKTKALAAIPPPVPPSATEPLDLGCSSCGTPLHDQGGPVEILPFLYLGSAYHAARRDMLDALGITALLNVSSDCPNHFEGHYQYKCIPVEDNHKADISSWFMEAIEYIDAVKDCRGRVLVHCQAGISRSATICLAYLMMKKRVRLEEAFEFVKQRRSIISPNFSFMGQLLQFESQVLATSCAAEAASPSGPLRERGKTPATPTSQFVFSFPVSVGVHSAPSSLPYLHSPITTSPSC",
# DUS5_HUMAN
"MKVTSLDGRQLRKMLRKEAAARCVVLDCRPYLAFAASNVRGSLNVNLNSVVLRRARGGAVSARYVLPDEAARARLLQEGGGGVAAVVVLDQGSRHWQKLREESAARVVLTSLLACLPAGPRVYFLKGGYETFYSEYPECCVDVKPISQEKIESERALISQCGKPVVNVSYRPAYDQGGPVEILPFLYLGSAYHASKCEFLANLHITALLNVSRRTSEACATHLHYKWIPVEDSHTADISSHFQEAIDFIDCVREKGGKVLVHCEAGISRSPTICMAYLMKTKQFRLKEAFDYIKQRRSMVSPNFGFMGQLLQYESEILPSTPNPQPPSCQGEAAGSSLIGHLQTLSPDMQGAYCTFPASVLAPVPTHSTVSELSRSPVATATSC",
# DUS10_HUMAN
"MPPSPLDDRVVVALSRPVRPQDLNLCLDSSYLGSANPGSNSHPPVIATTVVSLKAANLTYMPSSSGSARSLNCGCSSASCCTVATYDKDNQAQTQAIAAGTTTTAIGTSTTCPANQMVNNNENTGSLSPSSGVGSPVSGTPKQLASIKIIYPNDLAKKMTKCSKSHLPSQGPVIIDCRPFMEYNKSHIQGAVHINCADKISRRRLQQGKITVLDLISCREGKDSFKRIFSKEIIVYDENTNEPSRVMPSQPLHIVLESLKREGKEPLVLKGGLSSFKQNHENLCDNSLQLQECREVGGGASAASSLLPQPIPTTPDIENAELTPILPFLFLGNEQDAQDLDTMQRLNIGYVINVTTHLPLYHYEKGLFNYKRLPATDSNKQNLRQYFEEAFEFIEEAHQCGKGLLIHCQAGVSRSATIVIAYLMKHTRMTMTDAYKFVKGKRPIISPNLNFMGQLLEFEEDLNNGVTPRILTPKLMGVETVV",
# DUS11_HUMAN
"MSQWHHPRSGWGRRRDFSGRSSAKKKGGNHIPERWKDYLPVGQRMPGTRFIAFKVPLQKSFEKKLAPEECFSPLDLFNKIREQNEELGLIIDLTYTQRYYKPEDLPETVPYLKIFTVGHQVPDDETIFKFKHAVNGFLKENKDNDKLIGVHCTHGLNRTGYLICRYLIDVEGVRPDDAIELFNRCRGHCLERQNYIEDLQNGPIRKNWNSSVPRSSDFEDSAHLMQPVHNKPVKQGPRYNLHQIQGHSAPRHFHTQTQSLQQSVRKFSENPHVYQRHHLPPPGPPGEDYSHRRYSWNVKPNASRAAQDRRRWYPYNYSRLSYPACWEWTQ",
# DUS12_HUMAN
"MLEAPGPSDGCELSNPSASRVSCAGQMLEVQPGLYFGGAAAVAEPDHLREAGITAVLTVDSEEPSFKAGPGVEDLWRLFVPALDKPETDLLSHLDRCVAFIGQARAEGRAVLVHCHAGVSRSVAIITAFLMKTDQLPFEKAYEKLQILKPEAKMNEGFEWQLKLYQAMGYEVDTSSAIYKQYRLQKVTEKYPELQNLPQELFAVDPTTVSQGLKDEVLYKCRKCRRSLFRSSSILDHREGSGPIAFAHKRMTPSSMLTTGRQAQCTSYFIEPVQWMESALLGVMDGQLLCPKCSAKLGSFNWYGEQCSCGRWITPAFQIHKNRVDEMKILPVLGSQTGKI",
# DUS14_HUMAN
"MSSRGHSTLPRTLMAPRMISEGDIGGIAQITSSLFLGRGSVASNRHLLQARGITCIVNATIEIPNFNWPQFEYVKVPLADMPHAPIGLYFDTVADKIHSVSRKHGATLVHCAAGVSRSATLCIAYLMKFHNVCLLEAYNWVKARRPVIRPNVGFWRQLIDYERQLFGKSTVKMVQTPYGIVPDVYEKESRHLMPYWGI",
# DUS18_HUMAN
"MTAPSCAFPVQFRQPSVSGLSQITKSLYISNGVAANNKLMLSSNQITMVINVSVEVVNTLYEDIQYMQVPVADSPNSRLCDFFDPIADHIHSVEMKQGRTLLHCAAGVSRSAALCLAYLMKYHAMSLLDAHTWTKSCRPIIRPNSGFWEQLIHYEFQLFGKNTVHMVSSPVGMIPDIYEKEVRLMIPL",
# DUS21_HUMAN
"MTASASSFSSSQGVQQPSIYSFSQITRSLFLSNGVAANDKLLLSSNRITAIVNASVEVVNVFFEGIQYIKVPVTDARDSRLYDFFDPIADLIHTIDMRQGRTLLHCMAGVSRSASLCLAYLMKYHSMSLLDAHTWTKSRRPIIRPNNGFWEQLINYEFKLFNNNTVRMINSPVGNIPDIYEKDLRMMISM",
# DUS28_HUMAN
"MGPAEAGRRGAASPVPPPLVRVAPSLFLGSARAAGAEEQLARAGVTLCVNVSRQQPGPRAPGVAELRVPVFDDPAEDLLAHLEPTCAAMEAAVRAGGACLVYCKNGRSRSAAVCTAYLMRHRGLSLAKAFQMVKSARPVAEPNPGFWSQLQKYEEALQAQSCLQGEPPALGLGPEA",
# DUS15_HUMAN
"MTEGVLPGLYLGNFIDAKDLDQLGRNKITHIISIHESPQPLLQDITYLRIPVADTPEVPIKKHFKECINFIHCCRLNGGNCLVHCFAGISRSTTIVTAYVMTVTGLGWRDVLEAIKATRPIANPNPGFRQQLEEFGWASSQKGARHRTSKTSGAQCPPMTSATCLLAARVALLSAALVREATGRTAQRCRLSPRAAAERLLGPPPHVAAGWSPDPKYQICLCFGEEDPGPTQHPKEQLIMADVQVQLRPGSSSCTLSASTERPDGSSTPGNPDGITHLQCSCLHPKRAASSSCTR",
# DUS22_HUMAN
"MGNGMNKILPGLYIGNFKDARDAEQLSKNKVTHILSVHDSARPMLEGVKYLCIPAADSPSQNLTRHFKESIKFIHECRLRGESCLVHCLAGVSRSVTLVIAYIMTVTDFGWEDALHTVRAGRSCANPNVGFQRQLQEFEKHEVHQYRQWLKEEYGESPLQDAEEAKNILAAPGILKFWAFLRRL",
# DUS19_HUMAN
"MYSLNQEIKAFSRNNLRKQCTRVTTLTGKKIIETWKDARIHVVEEVEPSSGGGCGYVQDLSSDLQVGVIKPWLLLGSQDAAHDLDTLKKNKVTHILNVAYGVENAFLSDFTYKSISILDLPETNILSYFPECFEFIEEAKRKDGVVLVHCNAGVSRAAAIVIGFLMNSEQTSFTSAFSLVKNARPSICPNSGFMEQLRTYQEGKESNKCDRIQENSS",
# DUS23_HUMAN
"MGVQPPNFSWVLPGRLAGLALPRLPAHYQFLLDLGVRHLVSLTERGPPHSDSCPGLTLHRLRIPDFCPPAPDQIDRFVQIVDEANARGEAVGVHCALGFGRTGTMLACYLVKERGLAAGDAIAEIRRLRPGSIETYEQEKAVFQFYQRTK",
# DUS29_HUMAN
"MTSGEVKTSLKNAYSSAKRLSPKMEEEGEEEDYCTPGAFELERLFWKGSPQYTHVNEVWPKLYIGDEATALDRYRLQKAGFTHVLNAAHGRWNVDTGPDYYRDMDIQYHGVEADDLPTFDLSVFFYPAAAFIDRALSDDHSKILVHCVMGRSRSATLVLAYLMIHKDMTLVDAIQQVAKNRCVLPNRGFLKQLRELDKQLVQQRRRSQRQDGEEEDGREL",
# DS13A_HUMAN
"MAETSLPELGGEDKATPCPSILELEELLRAGKSSCSRVDEVWPNLFIGDAATANNRFELWKLGITHVLNAAHKGLYCQGGPDFYGSSVSYLGVPAHDLPDFDISAYFSSAADFIHRALNTPGAKVLVHCVVGVSRSATLVLAYLMLHQRLSLRQAVITVRQHRWVFPNRGFLHQLCRLDQQLRGAGQS",
# DS13B_HUMAN
"MDSLQKQDLRRPKIHGAVQASPYQPPTLASLQRLLWVRQAATLNHIDEVWPSLFLGDAYAARDKSKLIQLGITHVVNAAAGKFQVDTGAKFYRGMSLEYYGIEADDNPFFDLSVYFLPVARYIRAALSVPQGRVLVHCAMGVSRSATLVLAFLMICENMTLVEAIQTVQAHRNICPNSGFLRQLQVLDNRLGRETGRF",
# DUS26_HUMAN
"MCPGNWLWASMTFMARFSRSSSRSPVRTRGTLEEMPTVQHPFLNVFELERLLYTGKTACNHADEVWPGLYLGDQDMANNRRELRRLGITHVLNASHSRWRGTPEAYEGLGIRYLGVEAHDSPAFDMSIHFQTAADFIHRALSQPGGKILVHCAVGVSRSATLVLAYLMLYHHLTLVEAIKKVKDHRGIIPNRGFLRQLLALDRRLRQGLEA",
# STYL2_HUMAN
"MATRKDTEEEQVVPSEEDEANVRAVQAHYLRSPSPSQYSMVSDAETESIFMEPIHLSSAIAAKQIINEELKPPGVRADAECPGMLESAEQLLVEDLYNRVREKMDDTSLYNTPCVLDLQRALVQDRQEAPWNEVDEVWPNVFIAEKSVAVNKGRLKRLGITHILNAAHGTGVYTGPEFYTGLEIQYLGVEVDDFPEVDISQHFRKASEFLDEALLTYRGKVLVSSEMGISRSAVLVVAYLMIFHNMAILEALMTVRKKRAIYPNEGFLKQLRELNEKLMEEREEDYGREGGSAEAEEGEGTGSMLGARVHALTVEEEDDSASHLSGSSLGKATQASKPLTLIDEEEEEKLYEQWKKGQGLLSDKVPQDGGGWRSASSGQGGEELEDEDVERIIQEWQSRNERYQAEGYRRWGREEEKEEESDAGSSVGRRRRTLSESSAWESVSSHDIWVLKQQLELNRPDHGRRRRADSMSSESTWDAWNERLLEIEKEASRRYHAKSKREEAADRSSEAGSRVREDDEDSVGSEASSFYNFCSRNKDKLTALERWKIKRIQFGFHKKDLGAGDSSGEPGAEEAVGEKNPSDVSLTAYQAWKLKHQKKVGSENKEEVVELSKGEDSALAKKRQRRLELLERSRQTLEESQSMASWEADSSTASGSIPLSAFWSADPSVSADGDTTSVLSTQSHRSHLSQAASNIAGCSTSNPTTPLPNLPVGPGDTISIASIQNWIANVVSETLAQKQNEMLLLSRSPSVASMKAVPAASCLGDDQVSMLSGHSSSSLGGCLLPQSQARPSSDMQSVLSCNTTLSSPAESCRSKVRGTSKPIFSLFADNVDLKELGRKEKEMQMELREKMSEYKMEKLASDNKRSSLFKKKKVKEDEDDGVGDGDEDTDSAIGSFRYSSRSNSQKPETDTCSSLAVCDHYASGSRVGKEMDSSINKWLSGLRTEEKPPFQSDWSGSSRGKYTRSSLLRETESKSSSYKFSKSQSEEQDTSSYHEANGNSVRSTSRFSSSSTREGREMHKFSRSTYNETSSSREESPEPYFFRRTPESSEREESPEPQRPNWARSRDWEDVEESSKSDFSEFGAKRKFTQSFMRSEEEGEKERTENREEGRFASGRRSQYRRSTDREEEEEMDDEAIIAAWRRRQEETRTKLQKRRED",
# DUS3_HUMAN
"MSGSFELSVQDLNDLLSDGSGCYSLPSQPCNEVTPRIYVGNASVAQDIPKLQKLGITHVLNAAEGRSFMHVNTNANFYKDSGITYLGIKANDTQEFNLSAYFERAADFIDQALAQKNGRVLVHCREGYSRSPTLVIAYLMMRQKMDVKSALSIVRQNREIGPNDGFLAQLCQLNDRLAKEGKLKP",
# DUS6_HUMAN
"MIDTLRPVPFASEMAISKTVAWLNEQLELGNERLLLMDCRPQELYESSHIESAINVAIPGIMLRRLQKGNLPVRALFTRGEDRDRFTRRCGTDTVVLYDESSSDWNENTGGESVLGLLLKKLKDEGCRAFYLEGGFSKFQAEFSLHCETNLDGSCSSSSPPLPVLGLGGLRISSDSSSDIESDLDRDPNSATDSDGSPLSNSQPSFPVEILPFLYLGCAKDSTNLDVLEEFGIKYILNVTPNLPNLFENAGEFKYKQIPISDHWSQNLSQFFPEAISFIDEARGKNCGVLVHCLAGISRSVTVTVAYLMQKLNLSMNDAYDIVKMKKSNISPNFNFMGQLLDFERTLGLSSPCDNRVPAQQLYFTTPSNQNVYQVDSLQST",
# DUS7_HUMAN
"MKNQLRGPPARAHMSTSGAAAAGGTRAGSEPGAGSGSGAGTGAGAATGAGAMPCKSAEWLQEELEARGGASLLLLDCRPHELFESSHIETAINLAIPGLMLRRLRKGNLPIRSIIPNHADKERFATRCKAATVLLYDEATAEWQPEPGAPASVLGLLLQKLRDDGCQAYYLQGGFNKFQTEYSEHCETNVDSSSSPSSSPPTSVLGLGGLRISSDCSDGESDRELPSSATESDGSPVPSSQPAFPVQILPYLYLGCAKDSTNLDVLGKYGIKYILNVTPNLPNAFEHGGEFTYKQIPISDHWSQNLSQFFPEAISFIDEARSKKCGVLVHCLAGISRSVTVTVAYLMQKMNLSLNDAYDFVKRKKSNISPNFNFMGQLLDFERTLGLSSPCDNHASSEQLYFSTPTNHNLFPLNTLEST",
# DUS9_HUMAN
"MEGLGRSCLWLRRELSPPRPRLLLLDCRSRELYESARIGGALSVALPALLLRRLRRGSLSVRALLPGPPLQPPPPAPVLLYDQGGGRRRRGEAEAEAEEWEAESVLGTLLQKLREEGYLAYYLQGGFSRFQAECPHLCETSLAGRAGSSMAPVPGPVPVVGLGSLCLGSDCSDAESEADRDSMSCGLDSEGATPPPVGLRASFPVQILPNLYLGSARDSANLESLAKLGIRYILNVTPNLPNFFEKNGDFHYKQIPISDHWSQNLSRFFPEAIEFIDEALSQNCGVLVHCLAGVSRSVTVTVAYLMQKLHLSLNDAYDLVKRKKSNISPNFNFMGQLLDFERSLRLEERHSQEQGSGGQASAASNPPSFFTTPTSDGAFELAPT",
# DUS16_HUMAN
"MAHEMIGTQIVTERLVALLESGTEKVLLIDSRPFVEYNTSHILEAININCSKLMKRRLQQDKVLITELIQHSAKHKVDIDCSQKVVVYDQSSQDVASLSSDCFLTVLLGKLEKSFNSVHLLAGGFAEFSRCFPGLCEGKSTLVPTCISQPCLPVANIGPTRILPNLYLGCQRDVLNKELMQQNGIGYVLNASNTCPKPDFIPESHFLRVPVNDSFCEKILPWLDKSVDFIEKAKASNGCVLVHCLAGISRSATIAIAYIMKRMDMSLDEAYRFVKEKRPTISPNFNFLGQLLDYEKKIKNQTGASGPKSKLKLLHLEKPNEPVPAVSEGGQKSETPLSPPCADSATSEAAGQRPVHPASVPSVPSVQPSLLEDSPLVQALSGLHLSADRLEDSNKLKRSFSLDIKSVSYSASMAASLHGFSSSEDALEYYKPSTTLDGTNKLCQFSPVQELSEQTPETSPDKEEASIPKKLQTARPSDSQSKRLHSVRTSSSGTAQRSLLSPLHRSGSVEDNYHTSFLFGLSTSQQHLTKSAGLGLKGWHSDILAPQTSTPSLTSSWYFATESSHFYSASAIYGGSASYSAYSCSQLPTCGDQVYSVRRRQKPSDRADSRRSWHEESPFEKQFKRRSCQMEFGESIMSENRSREELGKVGSQSSFSGSMEIIEVS",
# DUS8_HUMAN
"MAGDRLPRKVMDAKKLASLLRGGPGGPLVIDSRSFVEYNSWHVLSSVNICCSKLVKRRLQQGKVTIAELIQPAARSQVEATEPQDVVVYDQSTRDASVLAADSFLSILLSKLDGCFDSVAILTGGFATFSSCFPGLCEGKPAALLPMSLSQPCLPVPSVGLTRILPHLYLGSQKDVLNKDLMTQNGISYVLNASNSCPKPDFICESRFMRVPINDNYCEKLLPWLDKSIEFIDKAKLSSCQVIVHCLAGISRSATIAIAYIMKTMGMSSDDAYRFVKDRRPSISPNFNFLGQLLEYERSLKLLAALQGDPGTPSGTPEPPPSPAAGAPLPRLPPPTSESAATGNAAAREGGLSAGGEPPAPPTPPATSALQQGLRGLHLSSDRLQDTNRLKRSFSLDIKSAYAPSRRPDGPGPPDPGEAPKLCKLDSPSGAALGLSSPSPDSPDAAPEARPRPRRRPRPPAGSPARSPAHSLGLNFGDAARQTPRHGLSALSAPGLPGPGQPAGPGAWAPPLDSPGTPSPDGPWCFSPEGAQGAGGVLFAPFGRAGAPGPGGGSDLRRREAARAEPRDARTGWPEEPAPETQFKRRSCQMEFEEGMVEGRARGEELAALGKQASFSGSVEVIEVS",
# TP4A1_HUMAN
"MARMNRPAPVEVTYKNMRFLITHNPTNATLNKFIEELKKYGVTTIVRVCEATYDTTLVEKEGIHVLDWPFDDGAPPSNQIVDDWLSLVKIKFREEPGCCIAVHCVAGLGRAPVLVALALIEGGMKYEDAVQFIRQKRRGAFNSKQLLYLEKYRPKMRLRFKDSNGHRNNCCIQ",
# TP4A2_HUMAN
"MNRPAPVEISYENMRFLITHNPTNATLNKFTEELKKYGVTTLVRVCDATYDKAPVEKEGIHVLDWPFDDGAPPPNQIVDDWLNLLKTKFREEPGCCVAVHCVAGLGRAPVLVALALIECGMKYEDAVQFIRQKRRGAFNSKQLLYLEKYRPKMRLRFRDTNGHCCVQ",
# TP4A3_HUMAN
"MARMNRPAPVEVSYKHMRFLITHNPTNATLSTFIEDLKKYGATTVVRVCEVTYDKTPLEKDGITVVDWPFDDGAPPPGKVVEDWLSLVKAKFCEAPGSCVAVHCVAGLGRAPVLVALALIESGMKYEDAIQFIRQKRRGAINSKQLTYLEKYRPKQRLRFKDPHTHKTRCCVM",
# PTPC1_HUMAN
"MAAGVLPQNEQPYSTLVNNSECVANMKGNLERPTPKYTKVGERLRHVIPGHMACSMACGGRACKYENPARWSEQEQAIKGVYSSWVTDNILAMARPSSELLEKYHIIDQFLSHGIKTIINLQRPGEHASCGNPLEQESGFTYLPEAFMEAGIYFYNFGWKDYGVASLTTILDMVKVMTFALQEGKVAIHCHAGLGRTGVLIACYLVFATRMTADQAIIFVRAKRPNSIQTRGQLLCVREFTQFLTPLRNIFSCCDPKAHAVTLPQYLIRQRHLLHGYEARLLKHVPKIIHLVCKLLLDLAENRPVMMKDVSEGPGLSAEIEKTMSEMVTMQLDKELLRHDSDVSNPPNPTAVAADFDNRGMIFSNEQQFDPLWKRRNVECLQPLTHLKRRLSYSDSDLKRAENLLEQGETPQTVPAQILVGHKPRQQKLISHCYIPQSPEPDLHKEALVRSTLSFWSQSKFGGLEGLKDNGSPIFHGRIIPKEAQQSGAFSADVSGSHSPGEPVSPSFANVHKDPNPAHQQVSHCQCKTHGVGSPGSVRQNSRTPRSPLDCGSSPKAQFLVEHETQDSKDLSEAASHSALQSELSAEARRILAAKALANLNESVEKEELKRKVEMWQKELNSRDGAWERICGERDPFILCSLMWSWVEQLKEPVITKEDVDMLVDRRADAAEALFLLEKGQHQTILCVLHCIVNLQTIPVDVEEAFLAHAIKAFTKVNFDSENGPTVYNTLKKIFKHTLEEKRKMTKDGPKPGL",
# PTPM1_HUMAN
"MAATALLEAGLARVLFYPTLLYTLFRGKVPGRAHRDWYHRIDPTVLLGALPLRSLTRQLVQDENVRGVITMNEEYETRFLCNSSQEWKRLGVEQLRLSTVDMTGIPTLDNLQKGVQFALKYQSLGQCVYVHCKAGRSRSATMVAAYLIQVHKWSPEEAVRAIAKIRSYIHIRPGQLDVLKEFHKQITARATKDGTFVISKT",
# MCE1_HUMAN
"MAHNKIPPRWLNCPRRGQPVAGRFLPLKTMLGPRYDSQVAEENRFHPSMLSNYLKSLKVKMGLLVDLTNTSRFYDRNDIEKEGIKYIKLQCKGHGECPTTENTETFIRLCERFNERNPPELIGVHCTHGFNRTGFLICAFLVEKMDWSIEAAVATFAQARPPGIYKGDYLKELFRRYGDIEEAPPPPLLPDWCFEDDEDEDEDEDGKKESEPGSSASFGKRRKERLKLGAIFLEGVTVKGVTQVTTQPKLGEVQQKCHQFCGWEGSGFPGAQPVSMDKQNIKLLDLKPYKVSWKADGTRYMMLIDGTNEVFMIDRDNSVFHVSNLEFPFRKDLRMHLSNTLLDGEMIIDRVNGQAVPRYLIYDIIKFNSQPVGDCDFNVRLQCIEREIISPRHEKMKTGLIDKTQEPFSVRNKPFFDICTSRKLLEGNFAKEVSHEMDGLIFQPTGKYKPGRCDDILKWKPPSLNSVDFRLKITRMGGEGLLPQNVGLLYVGGYERPFAQIKVTKELKQYDNKIIECKFENNSWVFMRQRTDKSFPNAYNTAMAVCNSISNPVTKEMLFEFIDRCTAASQGQKRKHHLDPDTELMPPPPPKRPRPLT",
# SSH1_HUMAN
"MALVTLQRSPTPSAASSSASNSELEAGSEEDRKLNLSLSESFFMVKGAALFLQQGSSPQGQRSLQHPHKHAGDLPQHLQVMINLLRCEDRIKLAVRLESAWADRVRYMVVVYSSGRQDTEENILLGVDFSSKESKSCTIGMVLRLWSDTKIHLDGDGGFSVSTAGRMHIFKPVSVQAMWSALQVLHKACEVARRHNYFPGGVALIWATYYESCISSEQSCINEWNAMQDLESTRPDSPALFVDKPTEGERTERLIKAKLRSIMMSQDLENVTSKEIRNELEKQMNCNLKELKEFIDNEMLLILGQMDKPSLIFDHLYLGSEWNASNLEELQGSGVDYILNVTREIDNFFPGLFAYHNIRVYDEETTDLLAHWNEAYHFINKAKRNHSKCLVHCKMGVSRSASTVIAYAMKEFGWPLEKAYNYVKQKRSITRPNAGFMRQLSEYEGILDASKQRHNKLWRQQTDSSLQQPVDDPAGPGDFLPETPDGTPESQLPFLDDAAQPGLGPPLPCCFRRLSDPLLPSPEDETGSLVHLEDPEREALLEEAAPPAEVHRPARQPQQGSGLCEKDVKKKLEFGSPKGRSGSLLQVEETEREEGLGAGRWGQLPTQLDQNLLNSENLNNNSKRSCPNGMEDDAIFGILNKVKPSYKSCADCMYPTASGAPEASRERCEDPNAPAICTQPAFLPHITSSPVAHLASRSRVPEKPASGPTEPPPFLPPAGSRRADTSGPGAGAALEPPASLLEPSRETPKVLPKSLLLKNSHCDKNPPSTEVVIKEESSPKKDMKPAKDLRLLFSNESEKPTTNSYLMQHQESIIQLQKAGLVRKHTKELERLKSVPADPAPPSRDGPASRLEASIPEESQDPAALHELGPLVMPSQAGSDEKSEAAPASLEGGSLKSPPPFFYRLDHTSSFSKDFLKTICYTPTSSSMSSNLTRSSSSDSIHSVRGKPGLVKQRTQEIETRLRLAGLTVSSPLKRSHSLAKLGSLTFSTEDLSSEADPSTVADSQDTTLSESSFLHEPQGTPRDPAATSKPSGKPAPENLKSPSWMSKS",
# SSH2_HUMAN
"MALVTVQRSPTPSTTSSPCASEADSGEEECRSQPRSISESFLTVKGAALFLPRGNGSSTPRISHRRNKHAGDLQQHLQAMFILLRPEDNIRLAVRLESTYQNRTRYMVVVSTNGRQDTEESIVLGMDFSSNDSSTCTMGLVLPLWSDTLIHLDGDGGFSVSTDNRVHIFKPVSVQAMWSALQSLHKACEVARAHNYYPGSLFLTWVSYYESHINSDQSSVNEWNAMQDVQSHRPDSPALFTDIPTERERTERLIKTKLREIMMQKDLENITSKEIRTELEMQMVCNLREFKEFIDNEMIVILGQMDSPTQIFEHVFLGSEWNASNLEDLQNRGVRYILNVTREIDNFFPGVFEYHNIRVYDEEATDLLAYWNDTYKFISKAKKHGSKCLVHCKMGVSRSASTVIAYAMKEYGWNLDRAYDYVKERRTVTKPNPSFMRQLEEYQGILLASKQRHNKLWRSHSDSDLSDHHEPICKPGLELNKKDITTSADQIAEVKTMESHPPIPPVFVEHMVPQDANQKGLCTKERMICLEFTSREFHAGQIEDELNLNDINGCSSGCCLNESKFPLDNCHASKALIQPGHVPEMANKFPDLTVEDLETDALKADMNVHLLPMEELTSPLKDPPMSPDPESPSPQPSCQTEISDFSTDRIDFFSALEKFVELSQETRSRSFSHSRMEELGGGRNESCRLSVVEVAPSKVTADDQRSSSLSNTPHASEESSMDEEQSKAISELVSPDIFMQSHSENAISVKEIVTEIESISQGVGQIQLKGDILPNPCHTPKKNSIHELLLERAQTPENKPGHMEQDEDSCTAQPELAKDSGMCNPEGCLTTHSSIADLEEGEPAEGEQELQGSGMHPGAKWYPGSVRRATLEFEERLRQEQEHHGAAPTCTSLSTRKNSKNDSSVADLAPKGKSDEAPPEHSFVLKEPEMSKGKGKYSGSEAGSLSHSEQNATVPAPRVLEFDHLPDPQEGPGSDTGTQQEGVLKDLRTVIPYQESETQAVPLPLPKRVEIIEYTHIVTSPNHTGPGSEIATSEKSGEQGLRKVNMEKSVTVLCTLDENLNRTLDPNQVSLHPQVLPLPHSSSPEHNRPTDHPTSILSSPEDRGSSLSTALETAAPFVSHTTHLLSASLDYLHPQTMVHLEGFTEQSSTTDEPSAEQVSWEESQESPLSSGSEVPYKDSQLSSADLSLISKLGDNTGELQEKMDPLPVACRLPHSSSSENIKSLSHSPGVVKERAKEIESRVVFQAGLTKPSQMRRSASLAKLGYLDLCKDCLPEREPASCESPHLKLLQPFLRTDSGMHAMEDQESLENPGAPHNPEPTKSFVEQLTTTECIVQSKPVERPLVQYAKEFGSSQQYLLPRAGLELTSSEGGLPVLQTQGLQCACPAPGLAVAPRQQHGRTHPLRRLKKANDKKRTTNPFYNTM",
# SSH3_HUMAN
"MALVTVSRSPPGSGASTPVGPWDQAVQRRSRLQRRQSFAVLRGAVLGLQDGGDNDDAAEASSEPTEKAPSEEELHGDQTDFGQGSQSPQKQEEQRQHLHLMVQLLRPQDDIRLAAQLEAPRPPRLRYLLVVSTREGEGLSQDETVLLGVDFPDSSSPSCTLGLVLPLWSDTQVYLDGDGGFSVTSGGQSRIFKPISIQTMWATLQVLHQACEAALGSGLVPGGSALTWASHYQERLNSEQSCLNEWTAMADLESLRPPSAEPGGSSEQEQMEQAIRAELWKVLDVSDLESVTSKEIRQALELRLGLPLQQYRDFIDNQMLLLVAQRDRASRIFPHLYLGSEWNAANLEELQRNRVTHILNMAREIDNFYPERFTYHNVRLWDEESAQLLPHWKETHRFIEAARAQGTHVLVHCKMGVSRSAATVLAYAMKQYECSLEQALRHVQELRPIARPNPGFLRQLQIYQGILTASRQSHVWEQKVGGVSPEEHPAPEVSTPFPPLPPEPEGGGEEKVVGMEESQAAPKEEPGPRPRINLRGVMRSISLLEPSLELESTSETSDMPEVFSSHESSHEEPLQPFPQLARTKGGQQVDRGPQPALKSRQSVVTLQGSAVVANRTQAFQEQEQGQGQGQGEPCISSTPRFRKVVRQASVHDSGEEGEA",
# STYX_HUMAN
"MEDVKLEFPSLPQCKEDAEEWTYPMRREMQEILPGLFLGPYSSAMKSKLPVLQKHGITHIICIRQNIEANFIKPNFQQLFRYLVLDIADNPVENIIRFFPMTKEFIDGSLQMGGKVLVHGNAGISRSAAFVIAYIMETFGMKYRDAFAYVQERRFCINPNAGFVHQLQEYEAIYLAKLTIQMMSPLQIERSLSVHSGTTGSLKRTHEEEDDFGTMQVATAQNG",
# STYL1_HUMAN
"MPGLLLCEPTELYNILNQATKLSRLTDPNYLCLLDVRSKWEYDESHVITALRVKKKNNEYLLPESVDLECVKYCVVYDNNSSTLEILLKDDDDDSDSDGDGKDLVPQAAIEYGRILTRLTHHPVYILKGGYERFSGTYHFLRTQKIIWMPQELDAFQPYPIEIVPGKVFVGNFSQACDPKIQKDLKIKAHVNVSMDTGPFFAGDADKLLHIRIEDSPEAQILPFLRHMCHFIEIHHHLGSVILIFSTQGISRSCAAIIAYLMHSNEQTLQRSWAYVKKCKNNMCPNRGLVSQLLEWEKTILGDSITNIMDPLY",
# MTM1_HUMAN
"MASASTSKYNSHSLENESIKRTSRDGVNRDLTEAVPRLPGETLITDKEVIYICPFNGPIKGRVYITNYRLYLRSLETDSSLILDVPLGVISRIEKMGGATSRGENSYGLDITCKDMRNLRFALKQEGHSRRDMFEILTRYAFPLAHSLPLFAFLNEEKFNVDGWTVYNPVEEYRRQGLPNHHWRITFINKCYELCDTYPALLVVPYRASDDDLRRVATFRSRNRIPVLSWIHPENKTVIVRCSQPLVGMSGKRNKDDEKYLDVIRETNKQISKLTIYDARPSVNAVANKATGGGYESDDAYHNAELFFLDIHNIHVMRESLKKVKDIVYPNVEESHWLSSLESTHWLEHIKLVLTGAIQVADKVSSGKSSVLVHCSDGWDRTAQLTSLAMLMLDSFYRSIEGFEILVQKEWISFGHKFASRIGHGDKNHTDADRSPIFLQFIDCVWQMSKQFPTAFEFNEQFLIIILDHLYSCRFGTFLFNCESARERQKVTERTVSLWSLINSNKEKFKNPFYTKEINRVLYPVASMRHLELWVNYYIRWNPRIKQQQPNPVEQRYMELLALRDEYIKRLEELQLANSAKLSDPPTSPSSPSQMMPHVQTHF",
# MTMR1_HUMAN
"MDRPAAAAAAGCEGGGGPNPGPAGGRRPPRAAGGATAGSRQPSVETLDSPTGSHVEWCKQLIAATISSQISGSVTSENVSRDYKALRDGNKLAQMEEAPLFPGESIKAIVKDVMYICPFMGAVSGTLTVTDFKLYFKNVERDPHFILDVPLGVISRVEKIGAQSHGDNSCGIEIVCKDMRNLRLAYKQEEQSKLGIFENLNKHAFPLSNGQALFAFSYKEKFPINGWKVYDPVSEYKRQGLPNESWKISKINSNYEFCDTYPAIIVVPTSVKDDDLSKVAAFRAKGRVPVLSWIHPESQATITRCSQPLVGPNDKRCKEDEKYLQTIMDANAQSHKLIIFDARQNSVADTNKTKGGGYESESAYPNAELVFLEIHNIHVMRESLRKLKEIVYPSIDEARWLSNVDGTHWLEYIRMLLAGAVRIADKIESGKTSVVVHCSDGWDRTAQLTSLAMLMLDSYYRTIKGFETLVEKEWISFGHRFALRVGHGNDNHADADRSPIFLQFVDCVWQMTRQFPSAFEFNELFLITILDHLYSCLFGTFLCNCEQQRFKEDVYTKTISLWSYINSQLDEFSNPFFVNYENHVLYPVASLSHLELWVNYYVRWNPRMRPQMPIHQNLKELLAVRAELQKRVEGLQREVATRAVSSSSERGSSPSHSATSVHTSV",
# MTMR2_HUMAN
"MEKSSSCESLGSQPAAARPPSVDSLSSASTSHSENSVHTKSASVVSSDSISTSADNFSPDLRVLRESNKLAEMEEPPLLPGENIKDMAKDVTYICPFTGAVRGTLTVTNYRLYFKSMERDPPFVLDASLGVINRVEKIGGASSRGENSYGLETVCKDIRNLRFAHKPEGRTRRSIFENLMKYAFPVSNNLPLFAFEYKEVFPENGWKLYDPLLEYRRQGIPNESWRITKINERYELCDTYPALLVVPANIPDEELKRVASFRSRGRIPVLSWIHPESQATITRCSQPMVGVSGKRSKEDEKYLQAIMDSNAQSHKIFIFDARPSVNAVANKAKGGGYESEDAYQNAELVFLDIHNIHVMRESLRKLKEIVYPNIEETHWLSNLESTHWLEHIKLILAGALRIADKVESGKTSVVVHCSDGWDRTAQLTSLAMLMLDGYYRTIRGFEVLVEKEWLSFGHRFQLRVGHGDKNHADADRSPVFLQFIDCVWQMTRQFPTAFEFNEYFLITILDHLYSCLFGTFLCNSEQQRGKENLPKRTVSLWSYINSQLEDFTNPLYGSYSNHVLYPVASMRHLELWVGYYIRWNPRMKPQEPIHNRYKELLAKRAELQKKVEELQREISNRSTSSSERASSPAQCVTPVQTVV",
# MTMRA_HUMAN
"MFSLKPPKPTFRSYLLPPPQTDDKINSEPKIKKLEPVLLPGEIVVNEVNFVRKCIATDTSQYDLWGKLICSNFKISFITDDPMPLQKFHYRNLLLGEHDVPLTCIEQIVTVNDHKRKQKVLGPNQKLKFNPTELIIYCKDFRIVRFRFDESGPESAKKVCLAIAHYSQPTDLQLLFAFEYVGKKYHNSANKINGIPSGDGGGGGGGGNGAGGGSSQKTPLFETYSDWDREIKRTGASGWRVCSINEGYMISTCLPEYIVVPSSLADQDLKIFSHSFVGRRMPLWCWSHSNGSALVRMALIKDVLQQRKIDQRICNAITKSHPQRSDVYKSDLDKTLPNIQEVQAAFVKLKQLCVNEPFEETEEKWLSSLENTRWLEYVRAFLKHSAELVYMLESKHLSVVLQEEEGRDLSCCVASLVQVMLDPYFRTITGFQSLIQKEWVMAGYQFLDRCNHLKRSEKESPLFLLFLDATWQLLEQYPAAFEFSETYLAVLYDSTRISLFGTFLFNSPHQRVKQSTEFAISKNIQLGDEKGLKFPSVWDWSLQFTAKDRTLFHNPFYIGKSTPCIQNGSVKSFKRTKKSYSSTLRGMPSALKNGIISDQELLPRRNSLILKPKPDPAQQTDSQNSDTEQYFREWFSKPANLHGVILPRVSGTHIKLWKLCYFRWVPEAQISLGGSITAFHKLSLLADEVDVLSRMLRQQRSGPLEACYGELGQSRMYFNASGPHHTDTSGTPEFLSSSFPFSPVGNLCRRSILGTPLSKFLSGAKIWLSTETLANED",
# MTMRB_HUMAN
"MWWGGRGQSFNIAPQKEEPEMGSVQENRMPEPRSRQPSSCLASRCLPGEQILAWAPGVRKGLEPELSGTLICTNFRVTFQPCGWQWNQDTPLNSEYDFALVNIGRLEAVSGLSRVQLLRPGSLHKFIPEEILIHGRDFRLLRVGFEAGGLEPQAFQVTMAIVQARAQSNQAQQYSGITLSKAGQGSGSRKPPIPLMETAEDWETERKKQAARGWRVSTVNERFDVATSLPRYFWVPNRILDSEVRRAFGHFHQGRGPRLSWHHPGGSDLLRCGGFYTASDPNKEDIRAVELMLQAGHSDVVLVDTMDELPSLADVQLAHLRLRALCLPDSSVAEDKWLSALEGTRWLDYVRACLRKASDISVLVTSRVRSVILQERGDRDLNGLLSSLVQLLSAPEARTLFGFQSLVQREWVAAGHPFLTRLGGTGASEEAPVFLLFLDCVWQLLQQFPADFEFSEFFLLALHDSVRVPDTLTFLRNTPWERGKQSGQLNSYTQVYTPGYSQPPAGNSFNLQLSVWDWDLRYSNAQILQFQNPGYDPEHCPDSWLPRPQPSFMVPGPPSSVWLFSRGALTPLNQLCPWRDSPSLLAVSSRWLPRPAISSESLADQEWGLPSHWGACPLPPGLLLPGYLGPQIRLWRRCYLRGRPEVQMGLSAPTISGLQDELSHLQELLRKWTPRISPEDHSKKRDPHTILNPTEIAGILKGRAEGDLG",
# MTMRC_HUMAN
"MLGKGVVGGGGGTKAPKPSFVSYVRPEEIHTNEKEVTEKEVTLHLLPGEQLLCEASTVLKYVQEDSCQHGVYGRLVCTDFKIAFLGDDESALDNDETQFKNKVIGENDITLHCVDQIYGVFDEKKKTLFGQLKKYPEKLIIHCKDLRVFQFCLRYTKEEEVKRIVSGIIHHTQAPKLLKRLFLFSYATAAQNNTVTDPKNHTVMFDTLKDWCWELERTKGNMKYKAVSVNEGYKVCERLPAYFVVPTPLPEENVQRFQGHGIPIWCWSCHNGSALLKMSALPKEQDDGILQIQKSFLDGIYKTIHRPPYEIVKTEDLSSNFLSLQEIQTAYSKFKQLFLIDNSTEFWDTDIKWFSLLESSSWLDIIRRCLKKAIEITECMEAQNMNVLLLEENASDLCCLISSLVQLMMDPHCRTRIGFQSLIQKEWVMGGHCFLDRCNHLRQNDKEEVPVFLLFLDCVWQLVHQHPPAFEFTETYLTVLSDSLYIPIFSTFFFNSPHQKDTNMGREGQDTQSKPLNLLTVWDWSVQFEPKAQTLLKNPLYVEKPKLDKGQRKGMRFKHQRQLSLPLTQSKSSPKRGFFREETDHLIKNLLGKRISKLINSSDELQDNFREFYDSWHSKSTDYHGLLLPHIEGPEIKVWAQRYLRWIPEAQILGGGQVATLSKLLEMMEEVQSLQEKIDERHHSQQAPQAEAPCLLRNSARLSSLFPFALLQRHSSKPVLPTSGWKALGDEDDLAKREDEFVDLGDV",
# MTMRE_HUMAN
"MAGARAAAAAASAGSSASSGNQPPQELGLGELLEEFSRTQYRAKDGSGTGGSKVERIEKRCLELFGRDYCFSVIPNTNGDICGHYPRHIVFLEYESSEKEKDTFESTVQVSKLQDLIHRSKMARCRGRFVCPVILFKGKHICRSATLAGWGELYGRSGYNYFFSGGADDAWADVEDVTEEDCALRSGDTHLFDKVRGYDIKLLRYLSVKYICDLMVENKKVKFGMNVTSSEKVDKAQRYADFTLLSIPYPGCEFFKEYKDRDYMAEGLIFNWKQDYVDAPLSIPDFLTHSLNIDWSQYQCWDLVQQTQNYLKLLLSLVNSDDDSGLLVHCISGWDRTPLFISLLRLSLWADGLIHTSLKPTEILYLTVAYDWFLFGHMLVDRLSKGEEIFFFCFNFLKHITSEEFSALKTQRRKSLPARDGGFTLEDICMLRRKDRGSTTSLGSDFSLVMESSPGATGSFTYEAVELVPAGAPTQAAWRKSHSSSPQSVLWNRPQPSEDRLPSQQGLAEARSSSSSSSNHSDNFFRMGSSPLEVPKPRSVDHPLPGSSLSTDYGSWQMVTGCGSIQERAVLHTDSSLPFSFPDELPNSCLLAALSDRETRLQEVRSAFLAAYSSTVGLRAVAPSPSGAIGGLLEQFARGVGLRSISSNAL",
# MTMR3_HUMAN
"MDEETRHSLECIQANQIFPRKQLIREDENLQVPFLELHGESTEFVGRAEDAIIALSNYRLHIKFKESLVNVPLQLIESVECRDIFQLHLTCKDCKVIRCQFSTFEQCQEWLKRLNNAIRPPAKIEDLFSFAYHAWCMEVYASEKEQHGDLCRPGEHVTSRFKNEVERMGFDMNNAWRISNINEKYKLCGSYPQELIVPAWITDKELESVSSFRSWKRIPAVIYRHQSNGAVIARCGQPEVSWWGWRNADDEHLVQSVAKACASDSRSSGSKLSTRNTSRDFPNGGDLSDVEFDSSLSNASGAESLAIQPQKLLILDARSYAAAVANRAKGGGCECPEYYPNCEVVFMGMANIHSIRRSFQSLRLLCTQMPDPGNWLSALESTKWLHHLSVLLKSALLVVHAVDQDQRPVLVHCSDGWDRTPQIVALAKLLLDPYYRTIEGFQVLVEMEWLDFGHKFADRCGHGENSDDLNERCPVFLQWLDCVHQLQRQFPCSFEFNEAFLVKLVQHTYSCLFGTFLCNNAKERGEKHTQERTCSVWSLLRAGNKAFKNLLYSSQSEAVLYPVCHVRNLMLWSAVYLPCPSPTTPVDDSCAPYPAPGTSPDDPPLSRLPKTRSYDNLTTACDNTVPLASRRCSDPSLNEKWQEHRRSLELSSLAGPGEDPLSADSLGKPTRVPGGAELSVAAGVAEGQMENILQEATKEESGVEEPAHRAGIEIQEGKEDPLLEKESRRKTPEASAIGLHQDPELGDAALRSHLDMSWPLFSQGISEQQSGLSVLLSSLQVPPRGEDSLEVPVEQFRIEEIAEGREEAVLPIPVDAKVGYGTSQSCSLLPSQVPFETRGPNVDSSTDMLVEDKVKSVSGPQGHHRSCLVNSGKDRLPQTMEPSPSETSLVERPQVGSVVHRTSLGSTLSLTRSPCALPLAECKEGLVCNGAPETENRASEQPPGLSTLQMYPTPNGHCANGEAGRSKDSLSRQLSAMSCSSAHLHSRNLHHKWLHSHSGRPSATSSPDQPSRSHLDDDGMSVYTDTIQQRLRQIESGHQQEVETLKKQVQELKSRLESQYLTSSLHFNGDFGDEVTSIPDSESNLDQNCLSRCSTEIFSEASWEQVDKQDTEMTRWLPDHLAAHCYACDSAFWLASRKHHCRNCGNVFCSSCCNQKVPVPSQQLFEPSRVCKSCYSSLHPTSSSIDLELDKPIAATSN",
# MTMR4_HUMAN
"MGEEGPPSLEYIQAKDLFPPKELVKEEENLQVPFTVLQGEGVEFLGRAADALIAISNYRLHIKFKDSVINVPLRMIDSVESRDMFQLHISCKDSKVVRCHFSTFKQCQEWLSRLSRATARPAKPEDLFAFAYHAWCLGLTEEDQHTHLCQPGEHIRCRQEAELARMGFDLQNVWRVSHINSNYKLCPSYPQKLLVPVWITDKELENVASFRSWKRIPVVVYRHLRNGAAIARCSQPEISWWGWRNADDEYLVTSIAKACALDPGTRATGGSLSTGNNDTSEACDADFDSSLTACSGVESTAAPQKLLILDARSYTAAVANRAKGGGCECEEYYPNCEVVFMGMANIHAIRNSFQYLRAVCSQMPDPSNWLSALESTKWLQHLSVMLKAAVLVANTVDREGRPVLVHCSDGWDRTPQIVALAKILLDPYYRTLEGFQVLVESDWLDFGHKFGDRCGHQENVEDQNEQCPVFLQWLDSVHQLLKQFPCLFEFNEAFLVKLVQHTYSCLYGTFLANNPCEREKRNIYKRTCSVWALLRAGNKNFHNFLYTPSSDMVLHPVCHVRALHLWTAVYLPASSPCTLGEENMDLYLSPVAQSQEFSGRSLDRLPKTRSMDDLLSACDTSSPLTRTSSDPNLNNHCQEVRVGLEPWHSNPEGSETSFVDSGVGGPQQTVGEVGLPPPLPSSQKDYLSNKPFKSHKSCSPSYKLLNTAVPREMKSNTSDPEIKVLEETKGPAPDPSAQDELGRTLDGIGEPPEHCPETEAVSALSKVISNKCDGVCNFPESSQNSPTGTPQQAQPDSMLGVPSKCVLDHSLSTVCNPPSAACQTPLDPSTDFLNQDPSGSVASISHQEQLSSVPDLTHGEEDIGKRGNNRNGQLLENPRFGKMPLELVRKPISQSQISEFSFLGSNWDSFQGMVTSFPSGEATPRRLLSYGCCSKRPNSKQMRATGPCFGGQWAQREGVKSPVCSSHSNGHCTGPGGKNQMWLSSHPKQVSSTKPVPLNCPSPVPPLYLDDDGLPFPTDVIQHRLRQIEAGYKQEVEQLRRQVRELQMRLDIRHCCAPPAEPPMDYEDDFTCLKESDGSDTEDFGSDHSEDCLSEASWEPVDKKETEVTRWVPDHMASHCYNCDCEFWLAKRRHHCRNCGNVFCAGCCHLKLPIPDQQLYDPVLVCNSCYEHIQVSRARELMSQQLKKPIATASS",
# MTMR5_HUMAN
"MARLADYFVLVAFGPHPRGSGEGQGQILQRFPEKDWEDNPFPQGIELFCQPSGWQLCPERNPPTFFVAVLTDINSERHYCACLTFWEPAEPSQQETTRVEDATEREEEGDEGGQTHLSPTAPAPSAQLFAPKTLVLVSRLDHTEVFRNSLGLIYAIHVEGLNVCLENVIGNLLTCTVPLAGGSQRTISLGAGDRQVIQTPLADSLPVSRCSVALLFRQLGITNVLSLFCAALTEHKVLFLSRSYQRLADACRGLLALLFPLRYSFTYVPILPAQLLEVLSTPTPFIIGVNAAFQAETQELLDVIVADLDGGTVTIPECVHIPPLPEPLQSQTHSVLSMVLDPELELADLAFPPPTTSTSSLKMQDKELRAVFLRLFAQLLQGYRWCLHVVRIHPEPVIRFHKAAFLGQRGLVEDDFLMKVLEGMAFAGFVSERGVPYRPTDLFDELVAHEVARMRADENHPQRVLRHVQELAEQLYKNENPYPAVAMHKVQRPGESSHLRRVPRPFPRLDEGTVQWIVDQAAAKMQGAPPAVKAERRTTVPSGPPMTAILERCSGLHVNSARRLEVVRNCISYVFEGKMLEAKKLLPAVLRALKGRAARRCLAQELHLHVQQNRAVLDHQQFDFVVRMMNCCLQDCTSLDEHGIAAALLPLVTAFCRKLSPGVTQFAYSCVQEHVVWSTPQFWEAMFYGDVQTHIRALYLEPTEDLAPAQEVGEAPSQEDERSALDVASEQRRLWPTLSREKQQELVQKEESTVFSQAIHYANRMSYLLLPLDSSKSRLLRERAGLGDLESASNSLVTNSMAGSVAESYDTESGFEDAETCDVAGAVVRFINRFVDKVCTESGVTSDHLKGLHVMVPDIVQMHIETLEAVQRESRRLPPIQKPKLLRPRLLPGEECVLDGLRVYLLPDGREEGAGGSAGGPALLPAEGAVFLTTYRVIFTGMPTDPLVGEQVVVRSFPVAALTKEKRISVQTPVDQLLQDGLQLRSCTFQLLKMAFDEEVGSDSAELFRKQLHKLRYPPDIRATFAFTLGSAHTPGRPPRVTKDKGPSLRTLSRNLVKNAKKTIGRQHVTRKKYNPPSWEHRGQPPPEDQEDEISVSEELEPSTLTPSSALKPSDRMTMSSLVERACCRDYQRLGLGTLSSSLSRAKSEPFRISPVNRMYAICRSYPGLLIVPQSVQDNALQRVSRCYRQNRFPVVCWRSGRSKAVLLRSGGLHGKGVVGLFKAQNAPSPGQSQADSSSLEQEKYLQAVVSSMPRYADASGRNTLSGFSSAHMGSHGKWGSVRTSGRSSGLGTDVGSRLAGRDALAPPQANGGPPDPGFLRPQRAALYILGDKAQLKGVRSDPLQQWELVPIEVFEARQVKASFKKLLKACVPGCPAAEPSPASFLRSLEDSEWLIQIHKLLQVSVLVVELLDSGSSVLVGLEDGWDITTQVVSLVQLLSDPFYRTLEGFRLLVEKEWLSFGHRFSHRGAHTLAGQSSGFTPVFLQFLDCVHQVHLQFPMEFEFSQFYLKFLGYHHVSRRFRTFLLDSDYERIELGLLYEEKGERRGQVPCRSVWEYVDRLSKRTPVFHNYMYAPEDAEVLRPYSNVSNLKVWDFYTEETLAEGPPYDWELAQGPPEPPEEERSDGGAPQSRRRVVWPCYDSCPRAQPDAISRLLEELQRLETELGQPAERWKDTWDRVKAAQRLEGRPDGRGTPSSLLVSTAPHHRRSLGVYLQEGPVGSTLSLSLDSDQSSGSTTSGSRQAARRSTSTLYSQFQTAESENRSYEGTLYKKGAFMKPWKARWFVLDKTKHQLRYYDHRVDTECKGVIDLAEVEAVAPGTPTMGAPKTVDEKAFFDVKTTRRVYNFCAQDVPSAQQWVDRIQSCLSDA",
# MTMRD_HUMAN
"MARLADYFIVVGYDHEKPGSGEGLGKIIQRFPQKDWDDTPFPQGIELFCQPGGWQLSRERKQPTFFVVVLTDIDSDRHYCSCLTFYEAEINLQGTKKEEIEGEAKVSGLIQPAEVFAPKSLVLVSRLYYPEIFRACLGLIYTVYVDSLNVSLESLIANLCACLVPAAGGSQKLFSLGAGDRQLIQTPLHDSLPITGTSVALLFQQLGIQNVLSLFCAVLTENKVLFHSASFQRLSDACRALESLMFPLKYSYPYIPILPAQLLEVLSSPTPFIIGVHSVFKTDVHELLDVIIADLDGGTIKIPECIHLSSLPEPLLHQTQSALSLILHPDLEVADHAFPPPRTALSHSKMLDKEVRAVFLRLFAQLFQGYRSCLQLIRIHAEPVIHFHKTAFLGQRGLVENDFLTKVLSGMAFAGFVSERGPPYRSCDLFDELVAFEVERIKVEENNPVKMIKHVRELAEQLFKNENPNPHMAFQKVPRPTEGSHLRVHILPFPEINEARVQELIQENVAKNQNAPPATRIEKKCVVPAGPPVVSIMDKVTTVFNSAQRLEVVRNCISFIFENKILETEKTLPAALRALKGKAARQCLTDELGLHVQQNRAILDHQQFDYIIRMMNCTLQDCSSLEEYNIAAALLPLTSAFYRKLAPGVSQFAYTCVQDHPIWTNQQFWETTFYNAVQEQVRSLYLSAKEDNHAPHLKQKDKLPDDHYQEKTAMDLAAEQLRLWPTLSKSTQQELVQHEESTVFSQAIHFANLMVNLLVPLDTSKNKLLRTSAPGDWESGSNSIVTNSIAGSVAESYDTESGFEDSENTDIANSVVRFITRFIDKVCTESGVTQDHIKSLHCMIPGIVAMHIETLEAVHRESRRLPPIQKPKILRPALLPGEEIVCEGLRVLLDPDGREEATGGLLGGPQLLPAEGALFLTTYRILFRGTPHDQLVGEQTVVRSFPIASITKEKKITMQNQLQQNMQEGLQITSASFQLIKVAFDEEVSPEVVEIFKKQLMKFRYPQSIFSTFAFAAGQTTPQIILPKQKEKNTSFRTFSKTIVKGAKRAGKMTIGRQYLLKKKTGTIVEERVNRPGWNEDDDVSVSDESELPTSTTLKASEKSTMEQLVEKACFRDYQRLGLGTISGSSSRSRPEYFRITASNRMYSLCRSYPGLLVVPQAVQDSSLPRVARCYRHNRLPVVCWKNSRSGTLLLRSGGFHGKGVVGLFKSQNSPQAAPTSSLESSSSIEQEKYLQALLNAVSVHQKLRGNSTLTVRPAFALSPGVWASLRSSTRLISSPTSFIDVGARLAGKDHSASFSNSSYLQNQLLKRQAALYIFGEKSQLRNFKVEFALNCEFVPVEFHEIRQVKASFKKLMRACIPSTIPTDSEVTFLKALGDSEWFPQLHRIMQLAVVVSEVLENGSSVLVCLEEGWDITAQVTSLVQLLSDPFYRTLEGFQMLVEKEWLSFGHKFSQRSSLTLNCQGSGFAPVFLQFLDCVHQVHNQYPTEFEFNLYYLKFLAFHYVSNRFKTFLLDSDYERLEHGTLFDDKGEKHAKKGVCIWECIDRMHKRSPIFFNYLYSPLEIEALKPNVNVSSLKKWDYYIEETLSTGPSYDWMMLTPKHFPSEDSDLAGEAGPRSQRRTVWPCYDDVSCTQPDALTSLFSEIEKLEHKLNQAPEKWQQLWERVTVDLKEEPRTDRSQRHLSRSPGIVSTNLPSYQKRSLLHLPDSSMGEEQNSSISPSNGVERRAATLYSQYTSKNDENRSFEGTLYKRGALLKGWKPRWFVLDVTKHQLRYYDSGEDTSCKGHIDLAEVEMVIPAGPSMGAPKHTSDKAFFDLKTSKRVYNFCAQDGQSAQQWMDKIQSCISDA",
# MTMR6_HUMAN
"MEHIRTTKVEQVKLLDRFSTSNKSLTGTLYLTATHLLFIDSHQKETWILHHHIASVEKLALTTSGCPLVIQCKNFRTVHFIVPRERDCHDIYNSLLQLSKQAKYEDLYAFSYNPKQNDSERLQGWQLIDLAEEYKRMGVPNSHWQLSDANRDYKICETYPRELYVPRIASKPIIVGSSKFRSKGRFPVLSYYHQDKEAAICRCSQPLSGFSARCLEDEHLLQAISKANPVNRYMYVMDTRPKLNAMANRAAGKGYENEDNYSNIRFQFVGIENIHVMRSSLQKLLEVNGTKGLSVNDFYSGLESSGWLRHIKAVMDAAIFLAKAITVENASVLVHCSDGWDRTSQVCSLGSLLLDSYYRTIKGFMVLIEKDWISFGHKFSERCGQLDGDPKEVSPVFTQFLECVWHLTEQFPQAFEFSEAFLLQIHEHIHSCQFGNFLGNCQKEREELKLKEKTYSLWPFLLEDQKKYLNPLYSSESHRFTVLEPNTVSFNFKFWRNMYHQFDRTLHPRQSVFNIIMNMNEQNKQLEKDIKDLESKIKQRKNKQTDGILTKELLHSVHPESPNLKTSLCFKEQTLLPVNDALRTIEGSSPADNRYSEYAEEFSKSEPAVVSLEYGVARMTC",
# MTMR7_HUMAN
"MEHIRTPKVENVRLVDRVSPKKAALGTLYLTATHVIFVENSPDPRKETWILHSQISTIEKQATTATGCPLLIRCKNFQIIQLIIPQERDCHDVYISLIRLARPVKYEELYCFSFNPMLDKEEREQGWVLIDLSEEYTRMGLPNHYWQLSDVNRDYRVCDSYPTELYVPKSATAHIIVGSSKFRSRRRFPVLSYYYKDNHASICRSSQPLSGFSARCLEDEQMLQAIRKANPGSDFVYVVDTRPKLNAMANRAAGKGYENEDNYSNIKFQFIGIENIHVMRNSLQKMLEVCELKSPSMSDFLWGLENSGWLRHIKAIMDAGIFIAKAVSEEGASVLVHCSDGWDRTAQVCSVASLLLDPHYRTLKGFMVLIEKDWISFGHKFNHRYGNLDGDPKEISPVIDQFIECVWQLMEQFPCAFEFNERFLIHIQHHIYSCQFGNFLCNSQKERRELKIQERTYSLWAHLWKNRADYLNPLFRADHSQTQGTLHLPTTPCNFMYKFWSGMYNRFEKGMQPRQSVTDYLMAVKEETQQLEEELEALEERLEKIQKVQLNCTKVKSKQSEPSKHSGFSTSDNSIANTPQDYSGNMKSFPSRSPSQGDEDSALILTQDNLKSSDPDLSANSDQESGVEDLSCRSPSGGEHAPSEDSGKDRDSDEAVFLTA",
# MTMR8_HUMAN
"MDHITVPKVENVKLVDRYVSKKPANGILYLTATHLIYVEASGAARKETWIALHHIATVEKLPITSLGCPLTLRCKNFRVAHFVLDSDLVCHEVYISLLKLSQPALPEDLYAFSYNPKSSKEMRESGWKLIDPISDFGRMGIPNRNWTITDANRNYEICSTYPPEIVVPKSVTLGTVVGSSKFRSKERVPVLSYLYKENNAAICRCSQPLSGFYTRCVDDELLLEAISQTNPGSQFMYVVDTRPKLNAMANRAAGKGYENEDNYANIRFRFMGIENIHVMRSSLQKLLEVCELKTPTMSEFLSGLESSGWLRHIKAIMDAGIFITKAVKVEKASVLVHCSDGWDRTAQVCSVASILLDPFYRTFKGLMILIEKEWISMGHKFSQRCGHLDGDSKEVSPIFTQFLDCIWQLMEQFPCAFEFNENFLLEIHDHVFSCQFGNFLGNCQKDREDLRVYEKTHSVWPFLVQRKPDFRNPLYKGFTMYGVLNPSTVPYNIQFWCGMYNRFDKGLQPKQSMLESLLEIKKQRAMLETDVHELEKKLKVRDEPPEEICTCSQLGNILSQHLGSPLTNPLGFMGINGDLNTLMENGTLSREGGLRAQMDQVKSQGADLHHNCCEIVGSLRAINISGDVGISEAMGISGDMCTFEATGFSKDLGICGAMDISEATGISGNLGISEARGFSGDMGILGDTGISKASTKEADYSKHQ",
# MTMR9_HUMAN
"MEFAELIKTPRVDNVVLHRPFYPAVEGTLCLTGHHLILSSRQDNTEELWLLHSNIDAIDKRFVGSLGTIIIKCKDFRIIQLDIPGMEECLNIASSIEALSTLDSITLMYPFFYRPMFEVIEDGWHSFLPEQEFELYSSATSEWRLSYVNKEFAVCPSYPPIVTVPKSIDDEALRKVATFRHGGRFPVLSYYHKKNGMVIMRSGQPLTGTNGRRCKEDEKLINATLRAGKRGYIIDTRSLNVAQQTRAKGGGFEQEAHYPQWRRIHKSIERYHILQESLIKLVEACNDQTHNMDRWLSKLEASNWLTHIKEILTTACLAAQCIDREGASILIHGTEGTDSTLQVTSLAQIILEPRSRTIRGFEALIEREWLQAGHPFQQRCAQSAYCNTKQKWEAPVFLLFLDCVWQILRQFPCSFEFNENFLIMLFEHAYASQFGTFLGNNESERCKLKLQQKTMSLWSWVNQPSELSKFTNPLFEANNLVIWPSVAPQSLPLWEGIFLRWNRSSKYLDEAYEEMVNIIEYNKELQAKVNILRRQLAELETEDGMQESP",
# PALD_HUMAN
"MGTTASTAQQTVSAGTPFEGLQGSGTMDSRHSVSIHSFQSTSLHNSKAKSIIPNKVAPVVITYNCKEEFQIHDELLKAHYTLGRLSDNTPEHYLVQGRYFLVRDVTEKMDVLGTVGSCGAPNFRQVQGGLTVFGMGQPSLSGFRRVLQKLQKDGHRECVIFCVREEPVLFLRADEDFVSYTPRDKQNLHENLQGLGPGVRVESLELAIRKEIHDFAQLSENTYHVYHNTEDLWGEPHAVAIHGEDDLHVTEEVYKRPLFLQPTYRYHRLPLPEQGSPLEAQLDAFVSVLRETPSLLQLRDAHGPPPALVFSCQMGVGRTNLGMVLGTLILLHRSGTTSQPEAAPTQAKPLPMEQFQVIQSFLRMVPQGRRMVEEVDRAITACAELHDLKEVVLENQKKLEGIRPESPAQGSGSRHSVWQRALWSLERYFYLILFNYYLHEQYPLAFALSFSRWLCAHPELYRLPVTLSSAGPVAPRDLIARGSLREDDLVSPDALSTVREMDVANFRRVPRMPIYGTAQPSAKALGSILAYLTDAKRRLRKVVWVSLREEAVLECDGHTYSLRWPGPPVAPDQLETLEAQLKAHLSEPPPGKEGPLTYRFQTCLTMQEVFSQHRRACPGLTYHRIPMPDFCAPREEDFDQLLEALRAALSKDPGTGFVFSCLSGQGRTTTAMVVAVLAFWHIQGFPEVGEEELVSVPDAKFTKGEFQVVMKVVQLLPDGHRVKKEVDAALDTVSETMTPMHYHLREIIICTYRQAKAAKEAQEMRRLQLRSLQYLERYVCLILFNAYLHLEKADSWQRPFSTWMQEVASKAGIYEILNELGFPELESGEDQPFSRLRYRWQEQSCSLEPSAPEDLL",
# AUXI_HUMAN
"MKDSENKGASSPDMEPSYGGGLFDMVKGGAGRLFSNLKDNLKDTLKDTSSRVIQSVTSYTKGDLDFTYVTSRIIVMSFPLDNVDIGFRNQVDDIRSFLDSRHLDHYTVYNLSPKSYRTAKFHSRVSECSWPIRQAPSLHNLFAVCRNMYNWLLQNPKNVCVVHCLDGRAASSILVGAMFIFCNLYSTPGPAIRLLYAKRPGIGLSPSHRRYLGYMCDLLADKPYRPHFKPLTIKSITVSPIPFFNKQRNGCRPYCDVLIGETKIYSTCTDFERMKEYRVQDGKIFIPLNITVQGDVVVSMYHLRSTIGSRLQAKVTNTQIFQLQFHTGFIPLDTTVLKFTKPELDACDVPEKYPQLFQVTLDVELQPHDKVIDLTPPWEHYCTKDVNPSILFSSHQEHQDTLALGGQAPIDIPPDNPRHYGQSGFFASLCWQDQKSEKSFCEEDHAALVNQESEQSDDELLTLSSPHGNANGDKPHGVKKPSKKQQEPAAPPPPEDVDLLGLEGSAMSNSFSPPAAPPTNSELLSDLFGGGGAAGPTQAGQSGVEDVFHPSGPASTQSTPRRSATSTSASPTLRVGEGATFDPFGAPSKPSGQDLLGSFLNTSSASSDPFLQPTRSPSPTVHASSTPAVNIQPDVSGGWDWHAKPGGFGMGSKSAATSPTGSSHGTPTHQSKPQTLDPFADLGTLGSSSFASKPTTPTGLGGGFPPLSSPQKASPQPMGGGWQQGGAYNWQQPQPKPQPSMPHSSPQNRPNYNVSFSAMPGGQNERGKGSSNLEGKQKAADFEDLLSGQGFNAHKDKKGPRTIAEMRKEEMAKEMDPEKLKILEWIEGKERNIRALLSTMHTVLWAGETKWKPVGMADLVTPEQVKKVYRKAVLVVHPDKATGQPYEQYAKMIFMELNDAWSEFENQGQKPLY",
# GAK_HUMAN
"MSLLQSALDFLAGPGSLGGASGRDQSDFVGQTVELGELRLRVRRVLAEGGFAFVYEAQDVGSGREYALKRLLSNEEEKNRAIIQEVCFMKKLSGHPNIVQFCSAASIGKEESDTGQAEFLLLTELCKGQLVEFLKKMESRGPLSCDTVLKIFYQTCRAVQHMHRQKPPIIHRDLKVENLLLSNQGTIKLCDFGSATTISHYPDYSWSAQRRALVEEEITRNTTPMYRTPEIIDLYSNFPIGEKQDIWALGCILYLLCFRQHPFEDGAKLRIVNGKYSIPPHDTQYTVFHSLIRAMLQVNPEERLSIAEVVHQLQEIAAARNVNPKSPITELLEQNGGYGSATLSRGPPPPVGPAGSGYSGGLALAEYDQPYGGFLDILRGGTERLFTNLKDTSSKVIQSVANYAKGDLDISYITSRIAVMSFPAEGVESALKNNIEDVRLFLDSKHPGHYAVYNLSPRTYRPSRFHNRVSECGWAARRAPHLHTLYNICRNMHAWLRQDHKNVCVVHCMDGRAASAVAVCSFLCFCRLFSTAEAAVYMFSMKRCPPGIWPSHKRYIEYMCDMVAEEPITPHSKPILVRAVVMTPVPLFSKQRSGCRPFCEVYVGDERVASTSQEYDKMRDFKIEDGKAVIPLGVTVQGDVLIVIYHARSTLGGRLQAKMASMKMFQIQFHTGFVPRNATTVKFAKYDLDACDIQEKYPDLFQVNLEVEVEPRDRPSREAPPWENSSMRGLNPKILFSSREEQQDILSKFGKPELPRQPGSTAQYDAGAGSPEAEPTDSDSPPSSSADASRFLHTLDWQEEKEAETGAENASSKESESALMEDRDESEVSDEGGSPISSEGQEPRADPEPPGLAAGLVQQDLVFEVETPAVLPEPVPQEDGVDLLGLHSEVGAGPAVPPQACKAPSSNTDLLSCLLGPPEAASQGPPEDLLSEDPLLLASPAPPLSVQSTPRGGPPAAADPFGPLLPSSGNNSQPCSNPDLFGEFLNSDSVTVPPSFPSAHSAPPPSCSADFLHLGDLPGEPSKMTASSSNPDLLGGWAAWTETAASAVAPTPATEGPLFSPGGQPAPCGSQASWTKSQNPDPFADLGDLSSGLQGSPAGFPPGGFIPKTATTPKGSSSWQTSRPPAQGASWPPQAKPPPKACTQPRPNYASNFSVIGAREERGVRAPSFAQKPKVSENDFEDLLSNQGFSSRSDKKGPKTIAEMRKQDLAKDTDPLKLKLLDWIEGKERNIRALLSTLHTVLWDGESRWTPVGMADLVAPEQVKKHYRRAVLAVHPDKAAGQPYEQHAKMIFMELNDAWSEFENQGSRPLF",
# PTEN_HUMAN
"MTAIIKEIVSRNKRRYQEDGFDLDLTYIYPNIIAMGFPAERLEGVYRNNIDDVVRFLDSKHKNHYKIYNLCAERHYDTAKFNCRVAQYPFEDHNPPQLELIKPFCEDLDQWLSEDDNHVAAIHCKAGKGRTGVMICAYLLHRGKFLKAQEALDFYGEVRTRDKKGVTIPSQRRYVYYYSYLLKNHLDYRPVALLFHKMMFETIPMFSGGTCNPQFVVCQLKVKIYSSNSGPTRREDKFMYFEFPQPLPVCGDIKVEFFHKQNKMLKKDKMFHFWVNTFFIPGPEETSEKVENGSLCDQEIDSICSIERADNDKEYLVLTLTKNDLDKANKDKANRYFSPNFKVKLYFTKTVEEPSNPEASSSTSVTPDVSDNEPDHYRYSDTTDSDPENEPFDEDQHTQITKV",
# TENS1_HUMAN
"MTWICLSCMLWPEDLEAPKTHRFKVKTFKKVKPCGICRQVITQEGCTCKVCSFSCHRKCQAKVAAPCVPPSNHELVPITTENAPKNVVDKGEGASRGGNTRKSLEDNGSTRVTPSVQPHLQPIRNMSVSRTMEDSCELDLVYVTERIIAVSFPSTANEENFRSNLREVAQMLKSKHGGNYLLFNLSERRPDITKLHAKVLEFGWPDLHTPALEKICSICKAMDTWLNADPHNVVVLHNKGNRGRIGVVIAAYMHYSNISASADQALDRFAMKRFYEDKIVPIGQPSQRRYVHYFSGLLSGSIKMNNKPLFLHHVIMHGIPNFESKGGCRPFLRIYQAMQPVYTSGIYNIPGDSQTSVCITIEPGLLLKGDILLKCYHKKFRSPARDVIFRVQFHTCAIHDLGVVFGKEDLDDAFKDDRFPEYGKVEFVFSYGPEKIQGMEHLENGPSVSVDYNTSDPLIRWDSYDNFSGHRDDGMEEVVGHTQGPLDGSLYAKVKKKDSLHGSTGAVNATRPTLSATPNHVEHTLSVSSDSGNSTASTKTDKTDEPVPGASSATAALSPQEKRELDRLLSGFGLEREKQGAMYHTQHLRSRPAGGSAVPSSGRHVVPAQVHVNGGALASERETDILDDELPNQDGHSAGSMGTLSSLDGVTNTSEGGYPEALSPLTNGLDKSYPMEPMVNGGGYPYESASRAGPAHAGHTAPMRPSYSAQEGLAGYQREGPHPAWPQPVTTSHYAHDPSGMFRSQSFSEAEPQLPPAPVRGGSSREAVQRGLNSWQQQQQQQQQPRPPPRQQERAHLESLVASRPSPQPLAETPIPSLPEFPRAASQQEIEQSIETLNMLMLDLEPASAAAPLHKSQSVPGAWPGASPLSSQPLSGSSRQSHPLTQSRSGYIPSGHSLGTPEPAPRASLESVPPGRSYSPYDYQPCLAGPNQDFHSKSPASSSLPAFLPTTHSPPGPQQPPASLPGLTAQPLLSPKEATSDPSRTPEEEPLNLEGLVAHRVAGVQAREKQPAEPPAPLRRRAASDGQYENQSPEATSPRSPGVRSPVQCVSPELALTIALNPGGRPKEPHLHSYKEAFEEMEGTSPSSPPPSGVRSPPGLAKTPLSALGLKPHNPADILLHPTGEPRSYVESVARTAVAGPRAQDSEPKSFSAPATQAYGHEIPLRNGTLGGSFVSPSPLSTSSPILSADSTSVGSFPSGESSDQGPRTPTQPLLESGFRSGSLGQPSPSAQRNYQSSSPLPTVGSSYSSPDYSLQHFSSSPESQARAQFSVAGVHTVPGSPQARHRTVGTNTPPSPGFGWRAINPSMAAPSSPSLSHHQMMGPPGTGFHGSTVSSPQSSAATTPGSPSLCRHPAGVYQVSGLHNKVATTPGSPSLGRHPGAHQGNLASGLHSNAIASPGSPSLGRHLGGSGSVVPGSPCLDRHVAYGGYSTPEDRRPTLSRQSSASGYQAPSTPSFPVSPAYYPGLSSPATSPSPDSAAFRQGSPTPALPEKRRMSVGDRAGSLPNYATINGKVSSPVASGMSSPSGGSTVSFSHTLPDFSKYSMPDNSPETRAKVKFVQDTSKYWYKPEISREQAIALLKDQEPGAFIIRDSHSFRGAYGLAMKVSSPPPTIMQQNKKGDMTHELVRHFLIETGPRGVKLKGCPNEPNFGSLSALVYQHSIIPLALPCKLVIPNRDPTDESKDSSGPANSTADLLKQGAACNVLFVNSVDMESLTGPQAISKATSETLAADPTPAATIVHFKVSAQGITLTDNQRKLFFRRHYPLNTVTFCDLDPQERKWMKTEGGAPAKLFGFVARKQGSTTDNACHLFAELDPNQPASAIVNFVSKVMLNAGQKR",
# TENS2_HUMAN
"MKSSGPVERLLRALGRRDSSRAASRPRKAEPHSFREKVFRKKPPVCAVCKVTIDGTGVSCRVCKVATHRKCEAKVTSACQALPPVELRRNTAPVRRIEHLGSTKSLNHSKQRSTLPRSFSLDPLMERRWDLDLTYVTERILAAAFPARPDEQRHRGHLRELAHVLQSKHRDKYLLFNLSEKRHDLTRLNPKVQDFGWPELHAPPLDKLCSICKAMETWLSADPQHVVVLYCKGNKGKLGVIVSAYMHYSKISAGADQALATLTMRKFCEDKVATELQPSQRRYISYFSGLLSGSIRMNSSPLFLHYVLIPMLPAFEPGTGFQPFLKIYQSMQLVYTSGVYHIAGPGPQQLCISLEPALLLKGDVMVTCYHKGGRGTDRTLVFRVQFHTCTIHGPQLTFPKDQLDEAWTDERFPFQASVEFVFSSSPEKIKGSTPRNDPSVSVDYNTTEPAVRWDSYENFNQHHEDSVDGSLTHTRGPLDGSPYAQVQRPPRQTPPAPSPEPPPPPMLSVSSDSGHSSTLTTEPAAESPGRPPPTAAERQELDRLLGGCGVASGGRGAGRETAILDDEEQPTVGGGPHLGVYPGHRPGLSRHCSCRQGYREPCGVPNGGYYRPEGTLERRRLAYGGYEGSPQGYAEASMEKRRLCRSLSEGLYPYPPEMGKPATGDFGYRAPGYREVVILEDPGLPALYPCPACEEKLALPTAALYGLRLEREAGEGWASEAGKPLLHPVRPGHPLPLLLPACGHHHAPMPDYSCLKPPKAGEEGHEGCSYTMCPEGRYGHPGYPALVTYSYGGAVPSYCPAYGRVPHSCGSPGEGRGYPSPGAHSPRAGSISPGSPPYPQSRKLSYEIPTEEGGDRYPLPGHLASAGPLASAESLEPVSWREGPSGHSTLPRSPRDAPCSASSELSGPSTPLHTSSPVQGKESTRRQDTRSPTSAPTQRLSPGEALPPVSQAGTGKAPELPSGSGPEPLAPSPVSPTFPPSSPSDWPQERSPGGHSDGASPRSPVPTTLPGLRHAPWQGPRGPPDSPDGSPLTPVPSQMPWLVASPEPPQSSPTPAFPLAASYDTNGLSQPPLPEKRHLPGPGQQPGPWGPEQASSPARGISHHVTFAPLLSDNVPQTPEPPTQESQSNVKFVQDTSKFWYKPHLSRDQAIALLKDKDPGAFLIRDSHSFQGAYGLALKVATPPPSAQPWKGDPVEQLVRHFLIETGPKGVKIKGCPSEPYFGSLSALVSQHSISPISLPCCLRIPSKDPLEETPEAPVPTNMSTAADLLRQGAACSVLYLTSVETESLTGPQAVARASSAALSCSPRPTPAVVHFKVSAQGITLTDNQRKLFFRRHYPVNSITFSSTDPQDRRWTNPDGTTSKIFGFVAKKPGSPWENVCHLFAELDPDQPAGAIVTFITKVLLGQRK",
# TENS3_HUMAN
"MEEGHGLDLTYITERIIAVSFPAGCSEESYLHNLQEVTRMLKSKHGDNYLVLNLSEKRYDLTKLNPKIMDVGWPELHAPPLDKMCTICKAQESWLNSNLQHVVVIHCRGGKGRIGVVISSYMHFTNVSASADQALDRFAMKKFYDDKVSALMQPSQKRYVQFLSGLLSGSVKMNASPLFLHFVILHGTPNFDTGGVCRPFLKLYQAMQPVYTSGIYNVGPENPSRICIVIEPAQLLKGDVMVKCYHKKYRSATRDVIFRLQFHTGAVQGYGLVFGKEDLDNASKDDRFPDYGKVELVFSATPEKIQGSEHLYNDHGVIVDYNTTDPLIRWDSYENLSADGEVLHTQGPVDGSLYAKVRKKSSSDPGIPGGPQAIPATNSPDHSDHTLSVSSDSGHSTASARTDKTEERLAPGTRRGLSAQEKAELDQLLSGFGLEDPGSSLKEMTDARSKYSGTRHVVPAQVHVNGDAALKDRETDILDDEMPHHDLHSVDSLGTLSSSEGPQSAHLGPFTCHKSSQNSLLSDGFGSNVGEDPQGTLVPDLGLGMDGPYERERTFGSREPKQPQPLLRKPSVSAQMQAYGQSSYSTQTWVRQQQMVVAHQYSFAPDGEARLVSRCPADNPGLVQAQPRVPLTPTRGTSSRVAVQRGVGSGPHPPDTQQPSPSKAFKPRFPGDQVVNGAGPELSTGPSPGSPTLDIDQSIEQLNRLILELDPTFEPIPTHMNALGSQANGSVSPDSVGGGLRASSRLPDTGEGPSRATGRQGSSAEQPLGGRLRKLSLGQYDNDAGGQLPFSKCAWGKAGVDYAPNLPPFPSPADVKETMTPGYPQDLDIIDGRILSSKESMCSTPAFPVSPETPYVKTALRHPPFSPPEPPLSSPASQHKGGREPRSCPETLTHAVGMSESPIGPKSTMLRADASSTPSFQQAFASSCTISSNGPGQRRESSSSAERQWVESSPKPMVSLLGSGRPTGSPLSAEFSGTRKDSPVLSCFPPSELQAPFHSHELSLAEPPDSLAPPSSQAFLGFGTAPVGSGLPPEEDLGALLANSHGASPTPSIPLTATGAADNGFLSHNFLTVAPGHSSHHSPGLQGQGVTLPGQPPLPEKKRASEGDRSLGSVSPSSSGFSSPHSGSTISIPFPNVLPDFSKASEAASPLPDSPGDKLVIVKFVQDTSKFWYKADISREQAIAMLKDKEPGSFIVRDSHSFRGAYGLAMKVATPPPSVLQLNKKAGDLANELVRHFLIECTPKGVRLKGCSNEPYFGSLTALVCQHSITPLALPCKLLIPERDPLEEIAESSPQTAANSAAELLKQGAACNVWYLNSVEMESLTGHQAIQKALSITLVQEPPPVSTVVHFKVSAQGITLTDNQRKLFFRRHYPVNSVIFCALDPQDRKWIKDGPSSKVFGFVARKQGSATDNVCHLFAEHDPEQPASAIVNFVSKVMIGSPKKV",
# TPTE_HUMAN
"MNESPDPTDLAGVIIELGPNDSPQTSEFKGATEEAPAKESPHTSEFKGAARVSPISESVLARLSKFEVEDAENVASYDSKIKKIVHSIVSSFAFGLFGVFLVLLDVTLILADLIFTDSKLYIPLEYRSISLAIALFFLMDVLLRVFVERRQQYFSDLFNILDTAIIVILLLVDVVYIFFDIKLLRNIPRWTHLLRLLRLIILLRIFHLFHQKRQLEKLIRRRVSENKRRYTRDGFDLDLTYVTERIIAMSFPSSGRQSFYRNPIKEVVRFLDKKHRNHYRVYNLCSERAYDPKHFHNRVVRIMIDDHNVPTLHQMVVFTKEVNEWMAQDLENIVAIHCKGGTDRTGTMVCAFLIASEICSTAKESLYYFGERRTDKTHSEKFQGVKTPSQKRYVAYFAQVKHLYNWNLPPRRILFIKHFIIYSIPRYVRDLKIQIEMEKKVVFSTISLGKCSVLDNITTDKILIDVFDGLPLYDDVKVQFFYSNLPTYYDNCSFYFWLHTSFIENNRLYLPKNELDNLHKQKARRIYPSDFAVEILFGEKMTSSDVVAGSD",
# TPTE2_HUMAN
"MNESPQTNEFKGTTEEAPAKESPHTSEFKGAALVSPISKSMLERLSKFEVEDAENVASYDSKIKKIVHSIVSSFAFGIFGVFLVLLDVTLLLADLIFTDSKLYIPLEYRSISLAIGLFFLMDVLLRVFVEGRQQYFSDLFNILDTAIIVIPLLVDVIYIFFDIKLLRNIPRWTHLVRLLRLIILIRIFHLLHQKRQLEKLMRRLVSENKRRYTRDGFDLDLTYVTERIIAMSFPSSGRQSFYRNPIEEVVRFLDKKHRNHYRVYNLCSERAYDPKHFHNRVSRIMIDDHNVPTLHEMVVFTKEVNEWMAQDLENIVAIHCKGGKGRTGTMVCALLIASEIFLTAEESLYYFGERRTNKTHSNKFQGVETPSQNRYVGYFAQVKHLYNWNLPPRRILFIKRFIIYSIRGDVCDLKVQVVMEKKVVFSSTSLGNCSILHDIETDKILINVYDGPPLYDDVKVQFFSSNLPKYYDNCPFFFWFNTSFIQNNRLCLPRNELDNPHKQKAWKIYPPEFAVEILFGEK",
# PTN1_HUMAN
"MEMEKEFEQIDKSGSWAAIYQDIRHEASDFPCRVAKLPKNKNRNRYRDVSPFDHSRIKLHQEDNDYINASLIKMEEAQRSYILTQGPLPNTCGHFWEMVWEQKSRGVVMLNRVMEKGSLKCAQYWPQKEEKEMIFEDTNLKLTLISEDIKSYYTVRQLELENLTTQETREILHFHYTTWPDFGVPESPASFLNFLFKVRESGSLSPEHGPVVVHCSAGIGRSGTFCLADTCLLLMDKRKDPSSVDIKKVLLEMRKFRMGLIQTADQLRFSYLAVIEGAKFIMGDSSVQDQWKELSHEDLEPPPEHIPPPPRPPKRILEPHNGKCREFFPNHQWVKEETQEDKDCPIKEEKGSPLNAAPYGIESMSQDTEVRSRVVGGSLRGAQAASPAKGEPSLPEKDEDHALSYWKPFLVNMCVATVLTAGAYLCYRFLFNSNT",
# PTN2_HUMAN
"MPTTIEREFEELDTQRRWQPLYLEIRNESHDYPHRVAKFPENRNRNRYRDVSPYDHSRVKLQNAENDYINASLVDIEEAQRSYILTQGPLPNTCCHFWLMVWQQKTKAVVMLNRIVEKESVKCAQYWPTDDQEMLFKETGFSVKLLSEDVKSYYTVHLLQLENINSGETRTISHFHYTTWPDFGVPESPASFLNFLFKVRESGSLNPDHGPAVIHCSAGIGRSGTFSLVDTCLVLMEKGDDINIKQVLLNMRKYRMGLIQTPDQLRFSYMAIIEGAKCIKGDSSIQKRWKELSKEDLSPAFDHSPNKIMTEKYNGNRIGLEEEKLTGDRCTGLSSKMQDTMEENSESALRKRIREDRKATTAQKVQQMKQRLNENERKRKRWLYWQPILTKMGFMSVILVGAFVGWTLFFQQNAL",
# PTN12_HUMAN
"MEQVEILRKFIQRVQAMKSPDHNGEDNFARDFMRLRRLSTKYRTEKIYPTATGEKEENVKKNRYKDILPFDHSRVKLTLKTPSQDSDYINANFIKGVYGPKAYVATQGPLANTVIDFWRMIWEYNVVIIVMACREFEMGRKKCERYWPLYGEDPITFAPFKISCEDEQARTDYFIRTLLLEFQNESRRLYQFHYVNWPDHDVPSSFDSILDMISLMRKYQEHEDVPICIHCSAGCGRTGAICAIDYTWNLLKAGKIPEEFNVFNLIQEMRTQRHSAVQTKEQYELVHRAIAQLFEKQLQLYEIHGAQKIADGVNEINTENMVSSIEPEKQDSPPPKPPRTRSCLVEGDAKEEILQPPEPHPVPPILTPSPPSAFPTVTTVWQDNDRYHPKPVLHMVSSEQHSADLNRNYSKSTELPGKNESTIEQIDKKLERNLSFEIKKVPLQEGPKSFDGNTLLNRGHAIKIKSASPCIADKISKPQELSSDLNVGDTSQNSCVDCSVTQSNKVSVTPPEESQNSDTPPRPDRLPLDEKGHVTWSFHGPENAIPIPDLSEGNSSDINYQTRKTVSLTPSPTTQVETPDLVDHDNTSPLFRTPLSFTNPLHSDDSDSDERNSDGAVTQNKTNISTASATVSAATSTESISTRKVLPMSIARHNIAGTTHSGAEKDVDVSEDSPPPLPERTPESFVLASEHNTPVRSEWSELQSQERSEQKKSEGLITSENEKCDHPAGGIHYEMCIECPPTFSDKREQISENPTEATDIGFGNRCGKPKGPRDPPSEWT",
# PTN18_HUMAN
"MSRSLDSARSFLERLEARGGREGAVLAGEFSDIQACSAAWKADGVCSTVAGSRPENVRKNRYKDVLPYDQTRVILSLLQEEGHSDYINGNFIRGVDGSLAYIATQGPLPHTLLDFWRLVWEFGVKVILMACREIENGRKRCERYWAQEQEPLQTGLFCITLIKEKWLNEDIMLRTLKVTFQKESRSVYQLQYMSWPDRGVPSSPDHMLAMVEEARRLQGSGPEPLCVHCSAGCGRTGVLCTVDYVRQLLLTQMIPPDFSLFDVVLKMRKQRPAAVQTEEQYRFLYHTVAQMFCSTLQNASPHYQNIKENCAPLYDDALFLRTPQALLAIPRPPGGVLRSISVPGSPGHAMADTYAVVQKRGAPAGAGSGTQTGTGTGTGARSAEEAPLYSKVTPRAQRPGAHAEDARGTLPGRVPADQSPAGSGAYEDVAGGAQTGGLGFNLRIGRPKGPRDPPAEWTRV",
# PTN22_HUMAN
"MDQREILQKFLDEAQSKKITKEEFANEFLKLKRQSTKYKADKTYPTTVAEKPKNIKKNRYKDILPYDYSRVELSLITSDEDSSYINANFIKGVYGPKAYIATQGPLSTTLLDFWRMIWEYSVLIIVMACMEYEMGKKKCERYWAEPGEMQLEFGPFSVSCEAEKRKSDYIIRTLKVKFNSETRTIYQFHYKNWPDHDVPSSIDPILELIWDVRCYQEDDSVPICIHCSAGCGRTGVICAIDYTWMLLKDGIIPENFSVFSLIREMRTQRPSLVQTQEQYELVYNAVLELFKRQMDVIRDKHSGTESQAKHCIPEKNHTLQADSYSPNLPKSTTKAAKMMNQQRTKMEIKESSSFDFRTSEISAKEELVLHPAKSSTSFDFLELNYSFDKNADTTMKWQTKAFPIVGEPLQKHQSLDLGSLLFEGCSNSKPVNAAGRYFNSKVPITRTKSTPFELIQQRETKEVDSKENFSYLESQPHDSCFVEMQAQKVMHVSSAELNYSLPYDSKHQIRNASNVKHHDSSALGVYSYIPLVENPYFSSWPPSGTSSKMSLDLPEKQDGTVFPSSLLPTSSTSLFSYYNSHDSLSLNSPTNISSLLNQESAVLATAPRIDDEIPPPLPVRTPESFIVVEEAGEFSPNVPKSLSSAVKVKIGTSLEWGGTSEPKKFDDSVILRPSKSVKLRSPKSELHQDRSSPPPPLPERTLESFFLADEDCMQAQSIETYSTSYPDTMENSTSSKQTLKTPGKSFTRSKSLKILRNMKKSICNSCPPNKPAESVQSNNSSSFLNFGFANRFSKPKGPRNPPPTWNI",
# PTN13_HUMAN
"MHVSLAEALEVRGGPLQEEEIWAVLNQSAESLQELFRKVSLADPAALGFIISPWSLLLLPSGSVSFTDENISNQDLRAFTAPEVLQNQSLTSLSDVEKIHIYSLGMTLYWGADYEVPQSQPIKLGDHLNSILLGMCEDVIYARVSVRTVLDACSAHIRNSNCAPSFSYVKHLVKLVLGNLSGTDQLSCNSEQKPDRSQAIRDRLRGKGLPTGRSSTSDVLDIQKPPLSHQTFLNKGLSKSMGFLSIKDTQDENYFKDILSDNSGREDSENTFSPYQFKTSGPEKKPIPGIDVLSKKKIWASSMDLLCTADRDFSSGETATYRRCHPEAVTVRTSTTPRKKEARYSDGSIALDIFGPQKMDPIYHTRELPTSSAISSALDRIRERQKKLQVLREAMNVEEPVRRYKTYHGDVFSTSSESPSIISSESDFRQVRRSEASKRFESSSGLPGVDETLSQGQSQRPSRQYETPFEGNLINQEIMLKRQEEELMQLQAKMALRQSRLSLYPGDTIKASMLDITRDPLREIALETAMTQRKLRNFFGPEFVKMTIEPFISLDLPRSILTKKGKNEDNRRKVNIMLLNGQRLELTCDTKTICKDVFDMVVAHIGLVEHHLFALATLKDNEYFFVDPDLKLTKVAPEGWKEEPKKKTKATVNFTLFFRIKFFMDDVSLIQHTLTCHQYYLQLRKDILEERMHCDDETSLLLASLALQAEYGDYQPEVHGVSYFRMEHYLPARVMEKLDLSYIKEELPKLHNTYVGASEKETELEFLKVCQRLTEYGVHFHRVHPEKKSQTGILLGVCSKGVLVFEVHNGVRTLVLRFPWRETKKISFSKKKITLQNTSDGIKHGFQTDNSKICQYLLHLCSYQHKFQLQMRARQSNQDAQDIERASFRSLNLQAESVRGFNMGRAISTGSLASSTLNKLAVRPLSVQAEILKRLSCSELSLYQPLQNSSKEKNDKASWEEKPREMSKSYHDLSQASLYPHRKNVIVNMEPPPQTVAELVGKPSHQMSRSDAESLAGVTKLNNSKSVASLNRSPERRKHESDSSSIEDPGQAYVLGMTMHSSGNSSSQVPLKENDVLHKRWSIVSSPEREITLVNLKKDAKYGLGFQIIGGEKMGRLDLGIFISSVAPGGPADLDGCLKPGDRLISVNSVSLEGVSHHAAIEILQNAPEDVTLVISQPKEKISKVPSTPVHLTNEMKNYMKKSSYMQDSAIDSSSKDHHWSRGTLRHISENSFGPSGGLREGSLSSQDSRTESASLSQSQVNGFFASHLGDQTWQESQHGSPSPSVISKATEKETFTDSNQSKTKKPGISDVTDYSDRGDSDMDEATYSSSQDHQTPKQESSSSVNTSNKMNFKTFSSSPPKPGDIFEVELAKNDNSLGISVTGGVNTSVRHGGIYVKAVIPQGAAESDGRIHKGDRVLAVNGVSLEGATHKQAVETLRNTGQVVHLLLEKGQSPTSKEHVPVTPQCTLSDQNAQGQGPEKVKKTTQVKDYSFVTEENTFEVKLFKNSSGLGFSFSREDNLIPEQINASIVRVKKLFPGQPAAESGKIDVGDVILKVNGASLKGLSQQEVISALRGTAPEVFLLLCRPPPGVLPEIDTALLTPLQSPAQVLPNSSKDSSQPSCVEQSTSSDENEMSDKSKKQCKSPSRRDSYSDSSGSGEDDLVTAPANISNSTWSSALHQTLSNMVSQAQSHHEAPKSQEDTICTMFYYPQKIPNKPEFEDSNPSPLPPDMAPGQSYQPQSESASSSSMDKYHIHHISEPTRQENWTPLKNDLENHLEDFELEVELLITLIKSEKGSLGFTVTKGNQRIGCYVHDVIQDPAKSDGRLKPGDRLIKVNDTDVTNMTHTDAVNLLRAASKTVRLVIGRVLELPRIPMLPHLLPDITLTCNKEELGFSLCGGHDSLYQVVYISDINPRSVAAIEGNLQLLDVIHYVNGVSTQGMTLEEVNRALDMSLPSLVLKATRNDLPVVPSSKRSAVSAPKSTKGNGSYSVGSCSQPALTPNDSFSTVAGEEINEISYPKGKCSTYQIKGSPNLTLPKESYIQEDDIYDDSQEAEVIQSLLDVVDEEAQNLLNENNAAGYSCGPGTLKMNGKLSEERTEDTDCDGSPLPEYFTEATKMNGCEEYCEEKVKSESLIQKPQEKKTDDDEITWGNDELPIERTNHEDSDKDHSFLTNDELAVLPVVKVLPSGKYTGANLKSVIRVLRGLLDQGIPSKELENLQELKPLDQCLIGQTKENRRKNRYKNILPYDATRVPLGDEGGYINASFIKIPVGKEEFVYIACQGPLPTTVGDFWQMIWEQKSTVIAMMTQEVEGEKIKCQRYWPNILGKTTMVSNRLRLALVRMQQLKGFVVRAMTLEDIQTREVRHISHLNFTAWPDHDTPSQPDDLLTFISYMRHIHRSGPIITHCSAGIGRSGTLICIDVVLGLISQDLDFDISDLVRCMRLQRHGMVQTEDQYIFCYQVILYVLTRLQAEEEQKQQPQLLK",
# PTN20_HUMAN
"MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFENKVNSEKVKLSLRNFPHNDYEDVFEEPSESGSDPSMWTARGPFRRDRWSSEDEEAAGPSQALSPLLSDTRKIVSEGELDQLAQIRPLIFNFHEQTAIKDCLKILEEKTAAYDIMQEFMALELKNLPGEFNSGNQPSNREKNRYRDILPYDSTRVPLGKSKDYINASYIRIVNCGEEYFYIATQGPLLSTIDDFWQMVLENNSNVIAMITREIEGGIIKCYHYWPISLKKPLELKHFRVFLENYQILQYFIIRMFQVVEKSTGTSHSVKQLQFTKWPDHGTPASADSFIKYIRYARKSHLTGPMVVHCSAGIGRTGVFLCVDVVFCAIVKNCSFNIMDIVAQMREQRSGMVQTKEQYHFCYDIVLEVLRKLLTLD",
# PTN14_HUMAN
"MPFGLKLRRTRRYNVLSKNCFVTRIRLLDSNVIECTLSVESTGQECLEAVAQRLELRETHYFGLWFLSKSQQARWVELEKPLKKHLDKFANEPLLFFGVMFYVPNVSWLQQEATRYQYYLQVKKDVLEGRLRCTLDQVIRLAGLAVQADFGDYNQFDSQDFLREYVLFPMDLALEEAVLEELTQKVAQEHKAHSGILPAEAELMYINEVERLDGFGQEIFPVKDNHGNCVHLGIFFMGIFVRNRIGRQAVIYRWNDMGNITHNKSTILVELINKEETALFHTDDIENAKYISRLFATRHKFYKQNKICTEQSNSPPPIRRQPTWSRSSLPRQQPYILPPVHVQCGEHYSETHTSQDSIFHGNEEALYCNSHNSLDLNYLNGTVTNGSVCSVHSVNSLNCSQSFIQASPVSSNLSIPGSDIMRADYIPSHRHSAIIVPSYRPTPDYETVMRQMKRGILHTDSQSQSLRNLNIINTHAYNQPEDLVYSQPEMRERHPYTVPYGPQGVYSNKLVSPSDQRNPKNNVVPSKPGASAISHTVSTPELANMQLQGSHNYSTAHMLKNYLFRPPPPYPRPRPATSTPDLASHRHKYVSGSSPDLVTRKVQLSVKTFQEDSSPVVHQSLQEVSEPLTATKHHGTVNKRHSLEVMNSMVRGMEAMTLKSLHLPMARRNTLREQGPPEEGSGSHEVPQLPQYHHKKTFSDATMLIHSSESEEEEEEAPESVPQIPMLREKMEYSAQLQAALARIPNKPPPEYPGPRKSVSNGALRQDQASLPPAMARARVLRHGPAKAISMSRTDPPAVNGASLGPSISEPDLTSVKERVKKEPVKERPVSEMFSLEDSIIEREMMIRNLEKQKMAGLEAQKRPLMLAALNGLSVARVSGREENRVDATRVPMDERFRTLKKKLEEGMVFTEYEQIPKKKANGIFSTAALPENAERSRIREVVPYEENRVELIPTKENNTGYINASHIKVVVGGAEWHYIATQGPLPHTCHDFWQMVWEQGVNVIAMVTAEEEGGRTKSHRYWPKLGSKHSSATYGKFKVTTKFRTDSVCYATTGLKVKHLLSGQERTVWHLQYTDWPDHGCPEDVQGFLSYLEEIQSVRRHTNSMLEGTKNRHPPIVVHCSAGVGRTGVLILSELMIYCLEHNEKVEVPMMLRLLREQRMFMIQTIAQYKFVYQVLIQFLQNSRLI",
# PTN21_HUMAN
"MPLPFGLKLKRTRRYTVSSKSCLVARIQLLNNEFVEFTLSVESTGQESLEAVAQRLELREVTYFSLWYYNKQNQRRWVDLEKPLKKQLDKYALEPTVYFGVVFYVPSVSQLQQEITRYQYYLQLKKDILEGSIPCTLEQAIQLAGLAVQADFGDFDQYESQDFLQKFALFPVGWLQDEKVLEEATQKVALLHQKYRGLTAPDAEMLYMQEVERMDGYGEESYPAKDSQGSDISIGACLEGIFVKHKNGRHPVVFRWHDIANMSHNKSFFALELANKEETIQFQTEDMETAKYIWRLCVARHKFYRLNQCNLQTQTVTVNPIRRRSSSRMSLPKPQPYVMPPPPQLHYNGHYTEPYASSQDNLFVPNQNGYYCHSQTSLDRAQIDLNGRIRNGSVYSAHSTNSLNNPQPYLQPSPMSSNPSITGSDVMRPDYLPSHRHSAVIPPSYRPTPDYETVMKQLNRGLVHAERQSHSLRNLNIGSSYAYSRPAALVYSQPEIREHAQLPSPAAAHCPFSLSYSFHSPSPYPYPAERRPVVGAVSVPELTNAQLQAQDYPSPNIMRTQVYRPPPPYPPPRPANSTPDLSRHLYISSSNPDLITRRVHHSVQTFQEDSLPVAHSLQEVSEPLTAARHAQLHKRNSIEVAGLSHGLEGLRLKERTLSASAAEVAPRAVSVGSQPSVFTERTQREGPEEAEGLRYGHKKSLSDATMLIHSSEEEEDEDFEEESGARAPPARAREPRPGLAQDPPGCPRVLLAGPLHILEPKAHVPDAEKRMMDSSPVRTTAEAQRPWRDGLLMPSMSESDLTTSGRYRARRDSLKKRPVSDLLSGKKNIVEGLPPLGGMKKTRVDAKKIGPLKLAALNGLSLSRVPLPDEGKEVATRATNDERCKILEQRLEQGMVFTEYERILKKRLVDGECSTARLPENAERNRFQDVLPYDDVRVELVPTKENNTGYINASHIKVSVSGIEWDYIATQGPLQNTCQDFWQMVWEQGIAIIAMVTAEEEGGREKSFRYWPRLGSRHNTVTYGRFKITTRFRTDSGCYATTGLKMKHLLTGQERTVWHLQYTDWPEHGCPEDLKGFLSYLEEIQSVRRHTNSTSDPQSPNPPLLVHCSAGVGRTGVVILSEIMIACLEHNEVLDIPRVLDMLRQQRMMLVQTLCQYTFVYRVLIQFLKSSRLI",
# PTN23_HUMAN
"MEAVPRMPMIWLDLKEAGDFHFQPAVKKFVLKNYGENPEAYNEELKKLELLRQNAVRVPRDFEGCSVLRKYLGQLHYLQSRVPMGSGQEAAVPVTWTEIFSGKSVAHEDIKYEQACILYNLGALHSMLGAMDKRVSEEGMKVSCTHFQCAAGAFAYLREHFPQAYSVDMSRQILTLNVNLMLGQAQECLLEKSMLDNRKSFLVARISAQVVDYYKEACRALENPDTASLLGRIQKDWKKLVQMKIYYFAAVAHLHMGKQAEEQQKFGERVAYFQSALDKLNEAIKLAKGQPDTVQDALRFTMDVIGGKYNSAKKDNDFIYHEAVPALDTLQPVKGAPLVKPLPVNPTDPAVTGPDIFAKLVPMAAHEASSLYSEEKAKLLREMMAKIEDKNEVLDQFMDSMQLDPETVDNLDAYSHIPPQLMEKCAALSVRPDTVRNLVQSMQVLSGVFTDVEASLKDIRDLLEEDELLEQKFQEAVGQAGAISITSKAELAEVRREWAKYMEVHEKASFTNSELHRAMNLHVGNLRLLSGPLDQVRAALPTPALSPEDKAVLQNLKRILAKVQEMRDQRVSLEQQLRELIQKDDITASLVTTDHSEMKKLFEEQLKKYDQLKVYLEQNLAAQDRVLCALTEANVQYAAVRRVLSDLDQKWNSTLQTLVASYEAYEDLMKKSQEGRDFYADLESKVAALLERTQSTCQAREAARQQLLDRELKKKPPPRPTAPKPLLPRREESEAVEAGDPPEELRSLPPDMVAGPRLPDTFLGSATPLHFPPSPFPSSTGPGPHYLSGPLPPGTYSGPTQLIQPRAPGPHAMPVAPGPALYPAPAYTPELGLVPRSSPQHGVVSSPYVGVGPAPPVAGLPSAPPPQFSGPELAMAVRPATTTVDSIQAPIPSHTAPRPNPTPAPPPPCFPVPPPQPLPTPYTYPAGAKQPIPAQHHFSSGIPAGFPAPRIGPQPQPHPQPHPSQAFGPQPPQQPLPLQHPHLFPPQAPGLLPPQSPYPYAPQPGVLGQPPPPLHTQLYPGPAQDPLPAHSGALPFPSPGPPQPPHPPLAYGPAPSTRPMGPQAAPLTIRGPSSAGQSTPSPHLVPSPAPSPGPGPVPPRPPAAEPPPCLRRGAAAADLLSSSPESQHGGTQSPGGGQPLLQPTKVDAAEGRRPQALRLIERDPYEHPERLRQLQQELEAFRGQLGDVGALDTVWRELQDAQEHDARGRSIAIARCYSLKNRHQDVMPYDSNRVVLRSGKDDYINASCVEGLSPYCPPLVATQAPLPGTAADFWLMVHEQKVSVIVMLVSEAEMEKQKVARYFPTERGQPMVHGALSLALSSVRSTETHVERVLSLQFRDQSLKRSLVHLHFPTWPELGLPDSPSNLLRFIQEVHAHYLHQRPLHTPIIVHCSSGVGRTGAFALLYAAVQEVEAGNGIPELPQLVRRMRQQRKHMLQEKLHLRFCYEAVVRHVEQVLQRHGVPPPCKPLASASISQKNHLPQDSQDLVLGGDVPISSIQATIAKLSIRPPGGLESPVASLPGPAEPPGLPPASLPESTPIPSSSPPPLSSPLPEAPQPKEEPPVPEAPSSGPPSSSLELLASLTPEAFSLDSSLRGKQRMSKHNFLQAHNGQGLRATRPSDDPLSLLDPLWTLNKT",
# PTN3_HUMAN
"MTSRLRALGGRINNIRTSELPKEKTRSEVICSIHFLDGVVQTFKVTKQDTGQVLLDMVHNHLGVTEKEYFGLQHDDDSVDSPRWLEASKAIRKQLKGGFPCTLHFRVRFFIPDPNTLQQEQTRHLYFLQLKMDICEGRLTCPLNSAVVLASYAVQSHFGDYNSSIHHPGYLSDSHFIPDQNEDFLTKVESLHEQHSGLKQSEAESCYINIARTLDFYGVELHSGRDLHNLDLMIGIASAGVAVYRKYICTSFYPWVNILKISFKRKKFFIHQRQKQAESREHIVAFNMLNYRSCKNLWKSCVEHHTFFQAKKLLPQEKNVLSQYWTMGSRNTKKSVNNQYCKKVIGGMVWNPAMRRSLSVEHLETKSLPSRSPPITPNWRSPRLRHEIRKPRHSSADNLANEMTYITETEDVFYTYKGSLAPQDSDSEVSQNRSPHQESLSENNPAQSYLTQKSSSSVSPSSNAPGSCSPDGVDQQLLDDFHRVTKGGSTEDASQYYCDKNDNGDSYLVLIRITPDEDGKFGFNLKGGVDQKMPLVVSRINPESPADTCIPKLNEGDQIVLINGRDISEHTHDQVVMFIKASRESHSRELALVIRRRAVRSFADFKSEDELNQLFPEAIFPMCPEGGDTLEGSMAQLKKGLESGTVLIQFEQLYRKKPGLAITFAKLPQNLDKNRYKDVLPYDTTRVLLQGNEDYINASYVNMEIPAANLVNKYIATQGPLPHTCAQFWQVVWDQKLSLIVMLTTLTERGRTKCHQYWPDPPDVMNHGGFHIQCQSEDCTIAYVSREMLVTNTQTGEEHTVTHLQYVAWPDHGVPDDSSDFLEFVNYVRSLRVDSEPVLVHCSAGIGRTGVLVTMETAMCLTERNLPIYPLDIVRKMRDQRAMMVQTSSQYKFVCEAILRVYEEGLVQMLDPS",
# PTN4_HUMAN
"MTSRFRLPAGRTYNVRASELARDRQHTEVVCNILLLDNTVQAFKVNKHDQGQVLLDVVFKHLDLTEQDYFGLQLADDSTDNPRWLDPNKPIRKQLKRGSPYSLNFRVKFFVSDPNKLQEEYTRYQYFLQIKQDILTGRLPCPSNTAALLASFAVQSELGDYDQSENLSGYLSDYSFIPNQPQDFEKEIAKLHQQHIGLSPAEAEFNYLNTARTLELYGVEFHYARDQSNNEIMIGVMSGGILIYKNRVRMNTFPWLKIVKISFKCKQFFIQLRKELHESRETLLGFNMVNYRACKNLWKACVEHHTFFRLDRPLPPQKNFFAHYFTLGSKFRYCGRTEVQSVQYGKEKANKDRVFARSPSKPLARKLMDWEVVSRNSISDDRLETQSLPSRSPPGTPNHRNSTFTQEGTRLRPSSVGHLVDHMVHTSPSEVFVNQRSPSSTQANSIVLESSPSQETPGDGKPPALPPKQSKKNSWNQIHYSHSQQDLESHINETFDIPSSPEKPTPNGGIPHDNLVLIRMKPDENGRFGFNVKGGYDQKMPVIVSRVAPGTPADLCVPRLNEGDQVVLINGRDIAEHTHDQVVLFIKASCERHSGELMLLVRPNAVYDVVEEKLENEPDFQYIPEKAPLDSVHQDDHSLRESMIQLAEGLITGTVLTQFDQLYRKKPGMTMSCAKLPQNISKNRYRDISPYDATRVILKGNEDYINANYINMEIPSSSIINQYIACQGPLPHTCTDFWQMTWEQGSSMVVMLTTQVERGRVKCHQYWPEPTGSSSYGCYQVTCHSEEGNTAYIFRKMTLFNQEKNESRPLTQIQYIAWPDHGVPDDSSDFLDFVCHVRNKRAGKEEPVVVHCSAGIGRTGVLITMETAMCLIECNQPVYPLDIVRTMRDQRAMMIQTPSQYRFVCEAILKVYEEGFVKPLTTSTNK",
# PTN5_HUMAN
"MNYEGARSERENHAADDSEGGALDMCCSERLPGLPQPIVMEALDEAEGLQDSQREMPPPPPPSPPSDPAQKPPPRGAGSHSLTVRSSLCLFAASQFLLACGVLWFSGYGHIWSQNATNLVSSLLTLLKQLEPTAWLDSGTWGVPSLLLVFLSVGLVLVTTLVWHLLRTPPEPPTPLPPEDRRQSVSRQPSFTYSEWMEEKIEDDFLDLDPVPETPVFDCVMDIKPEADPTSLTVKSMGLQERRGSNVSLTLDMCTPGCNEEGFGYLMSPREESAREYLLSASRVLQAEELHEKALDPFLLQAEFFEIPMNFVDPKEYDIPGLVRKNRYKTILPNPHSRVCLTSPDPDDPLSSYINANYIRGYGGEEKVYIATQGPIVSTVADFWRMVWQEHTPIIVMITNIEEMNEKCTEYWPEEQVAYDGVEITVQKVIHTEDYRLRLISLKSGTEERGLKHYWFTSWPDQKTPDRAPPLLHLVREVEEAAQQEGPHCAPIIVHCSAGIGRTGCFIATSICCQQLRQEGVVDILKTTCQLRQDRGGMIQTCEQYQFVHHVMSLYEKQLSHQSPE",
# PTN7_HUMAN
"MVQAHGGRSRAQPLTLSLGAAMTQPPPEKTPAKKHVRLQERRGSNVALMLDVRSLGAVEPICSVNTPREVTLHFLRTAGHPLTRWALQRQPPSPKQLEEEFLKIPSNFVSPEDLDIPGHASKDRYKTILPNPQSRVCLGRAQSQEDGDYINANYIRGYDGKEKVYIATQGPMPNTVSDFWEMVWQEEVSLIVMLTQLREGKEKCVHYWPTEEETYGPFQIRIQDMKECPEYTVRQLTIQYQEERRSVKHILFSAWPDHQTPESAGPLLRLVAEVEESPETAAHPGPIVVHCSAGIGRTGCFIATRIGCQQLKARGEVDILGIVCQLRLDRGGMIQTAEQYQFLHHTLALYAGQLPEEPSP",
# PTPRR_HUMAN
"MRRAVCFPALCLLLNLHAAGCFSGNNDHFLAINQKKSGKPVFIYKHSQDIEKSLDIAPQKIYRHSYHSSSEAQVSKRHQIVNSAFPRPAYDPSLNLLAMDGQDLEVENLPIPAANVIVVTLQMDVNKLNITLLRIFRQGVAAALGLLPQQVHINRLIGKKNSIELFVSPINRKTGISDALPSEEVLRSLNINVLHQSLSQFGITEVSPEKNVLQGQHEADKIWSKEGFYAVVIFLSIFVIIVTCLMILYRLKERFQLSLRQDKEKNQEIHLSPITLQPALSEAKTVHSMVQPEQAPKVLNVVVDPQGRGAPEIKATTATSVCPSPFKMKPIGLQERRGSNVSLTLDMSSLGNIEPFVSIPTPREKVAMEYLQSASRILTRSQLRDVVASSHLLQSEFMEIPMNFVDPKEIDIPRHGTKNRYKTILPNPLSRVCLRPKNVTDSLSTYINANYIRGYSGKEKAFIATQGPMINTVDDFWQMVWQEDSPVIVMITKLKEKNEKCVLYWPEKRGIYGKVEVLVISVNECDNYTIRNLVLKQGSHTQHVKHYWYTSWPDHKTPDSAQPLLQLMLDVEEDRLASQGRGPVVVHCSAGIGRTGCFIATSIGCQQLKEEGVVDALSIVCQLRMDRGGMVQTSEQYEFVHHALCLYESRLSAETVQ",
# PTN11_HUMAN
"MTSRRWFHPNITGVEAENLLLTRGVDGSFLARPSKSNPGDFTLSVRRNGAVTHIKIQNTGDYYDLYGGEKFATLAELVQYYMEHHGQLKEKNGDVIELKYPLNCADPTSERWFHGHLSGKEAEKLLTEKGKHGSFLVRESQSHPGDFVLSVRTGDDKGESNDGKSKVTHVMIRCQELKYDVGGGERFDSLTDLVEHYKKNPMVETLGTVLQLKQPLNTTRINAAEIESRVRELSKLAETTDKVKQGFWEEFETLQQQECKLLYSRKEGQRQENKNKNRYKNILPFDHTRVVLHDGDPNEPVSDYINANIIMPEFETKCNNSKPKKSYIATQGCLQNTVNDFWRMVFQENSRVIVMTTKEVERGKSKCVKYWPDEYALKEYGVMRVRNVKESAAHDYTLRELKLSKVGQGNTERTVWQYHFRTWPDHGVPSDPGGVLDFLEEVHHKQESIMDAGPVVVHCSAGIGRTGTFIVIDILIDIIREKGVDCDIDVPKTIQMVRSQRSGMVQTEAQYRFIYMAVQHYIETLQRRIEEEQKSKRKGHEYTNIKYSLADQTSGDQSPLPPCTPTPPCAEMREDSARVYENVGLMQQQKSFR",
# PTN6_HUMAN
"MVRWFHRDLSGLDAETLLKGRGVHGSFLARPSRKNQGDFSLSVRVGDQVTHIRIQNSGDFYDLYGGEKFATLTELVEYYTQQQGVLQDRDGTIIHLKYPLNCSDPTSERWYHGHMSGGQAETLLQAKGEPWTFLVRESLSQPGDFVLSVLSDQPKAGPGSPLRVTHIKVMCEGGRYTVGGLETFDSLTDLVEHFKKTGIEEASGAFVYLRQPYYATRVNAADIENRVLELNKKQESEDTAKAGFWEEFESLQKQEVKNLHQRLEGQRPENKGKNRYKNILPFDHSRVILQGRDSNIPGSDYINANYIKNQLLGPDENAKTYIASQGCLEATVNDFWQMAWQENSRVIVMTTREVEKGRNKCVPYWPEVGMQRAYGPYSVTNCGEHDTTEYKLRTLQVSPLDNGDLIREIWHYQYLSWPDHGVPSEPGGVLSFLDQINQRQESLPHAGPIIVHCSAGIGRTGTIIVIDMLMENISTKGLDCDIDIQKTIQMVRAQRSGMVQTEAQYKFIYVAIAQFIETTKKKLEVLQSQKGQESEYGNITYPPAMKNAHAKASRTSSKHKEDVYENLHTKNKREEKVKKQRSADKEKSKGSLKRK",
# PTN9_HUMAN
"MEPATAPRPDMAPELTPEEEQATKQFLEEINKWTVQYNVSPLSWNVAVKFLMARKFDVLRAIELFHSYRETRRKEGIVKLKPHEEPLRSEILSGKFTILNVRDPTGASIALFTARLHHPHKSVQHVVLQALFYLLDRAVDSFETQRNGLVFIYDMCGSNYANFELDLGKKVLNLLKGAFPARLKKVLIVGAPIWFRVPYSIISLLLKDKVRERIQILKTSEVTQHLPRECLPENLGGYVKIDLATWNFQFLPQVNGHPDPFDEIILFSLPPALDWDSVHVPGPHAMTIQELVDYVNARQKQGIYEEYEDIRRENPVGTFHCSMSPGNLEKNRYGDVPCLDQTRVKLTKRSGHTQTDYINASFMDGYKQKNAYIGTQGPLENTYRDFWLMVWEQKVLVIVMTTRFEEGGRRKCGQYWPLEKDSRIRFGFLTVTNLGVENMNHYKKTTLEIHNTEERQKRQVTHFQFLSWPDYGVPSSAASLIDFLRVVRNQQSLAVSNMGARSKGQCPEPPIVVHCSAGIGRTGTFCSLDICLAQLEELGTLNVFQTVSRMRTQRAFSIQTPEQYYFCYKAILEFAEKEGMVSSGQNLLAVESQ",
# PTPRA_HUMAN
"MDSWFILVLLGSGLICVSANNATTVAPSVGITRLINSSTAEPVKEEAKTSNPTSSLTSLSVAPTFSPNITLGPTYLTTVNSSDSDNGTTRTASTNSIGITISPNGTWLPDNQFTDARTEPWEGNSSTAATTPETFPPSGNSDSKDRRDETPIIAVMVALSSLLVIVFIIIVLYMLRFKKYKQAGSHSNSFRLSNGRTEDVEPQSVPLLARSPSTNRKYPPLPVDKLEEEINRRMADDNKLFREEFNALPACPIQATCEAASKEENKEKNRYVNILPYDHSRVHLTPVEGVPDSDYINASFINGYQEKNKFIAAQGPKEETVNDFWRMIWEQNTATIVMVTNLKERKECKCAQYWPDQGCWTYGNIRVSVEDVTVLVDYTVRKFCIQQVGDMTNRKPQRLITQFHFTSWPDFGVPFTPIGMLKFLKKVKACNPQYAGAIVVHCSAGVGRTGTFVVIDAMLDMMHTERKVDVYGFVSRIRAQRCQMVQTDMQYVFIYQALLEHYLYGDTELEVTSLETHLQKIYNKIPGTSNNGLEEEFKKLTSIKIQNDKMRTGNLPANMKKNRVLQIIPYEFNRVIIPVKRGEENTDYVNASFIDGYRQKDSYIASQGPLLHTIEDFWRMIWEWKSCSIVMLTELEERGQEKCAQYWPSDGLVSYGDITVELKKEEECESYTVRDLLVTNTRENKSRQIRQFHFHGWPEVGIPSDGKGMISIIAAVQKQQQQSGNHPITVHCSAGAGRTGTFCALSTVLERVKAEGILDVFQTVKSLRLQRPHMVQTLEQYEFCYKVVQEYIDAFSDYANFK",
# PTPRE_HUMAN
"MEPLCPLLLVGFSLPLARALRGNETTADSNETTTTSGPPDPGASQPLLAWLLLPLLLLLLVLLLAAYFFRFRKQRKAVVSTSDKKMPNGILEEQEQQRVMLLSRSPSGPKKYFPIPVEHLEEEIRIRSADDCKQFREEFNSLPSGHIQGTFELANKEENREKNRYPNILPNDHSRVILSQLDGIPCSDYINASYIDGYKEKNKFIAAQGPKQETVNDFWRMVWEQKSATIVMLTNLKERKEEKCHQYWPDQGCWTYGNIRVCVEDCVVLVDYTIRKFCIQPQLPDGCKAPRLVSQLHFTSWPDFGVPFTPIGMLKFLKKVKTLNPVHAGPIVVHCSAGVGRTGTFIVIDAMMAMMHAEQKVDVFEFVSRIRNQRPQMVQTDMQYTFIYQALLEYYLYGDTELDVSSLEKHLQTMHGTTTHFDKIGLEEEFRKLTNVRIMKENMRTGNLPANMKKARVIQIIPYDFNRVILSMKRGQEYTDYINASFIDGYRQKDYFIATQGPLAHTVEDFWRMIWEWKSHTIVMLTEVQEREQDKCYQYWPTEGSVTHGEITIEIKNDTLSEAISIRDFLVTLNQPQARQEEQVRVVRQFHFHGWPEIGIPAEGKGMIDLIAAVQKQQQQTGNHPITVHCSAGAGRTGTFIALSNILERVKAEGLLDVFQAVKSLRLQRPHMVQTLEQYEFCYKVVQDFIDIFSDYANFK",
# PTPRB_HUMAN
"MLSHGAGLALWITLSLLQTGLAEPERCNFTLAESKASSHSVSIQWRILGSPCNFSLIYSSDTLGAALCPTFRIDNTTYGCNLQDLQAGTIYNFRIISLDEERTVVLQTDPLPPARFGVSKEKTTSTSLHVWWTPSSGKVTSYEVQLFDENNQKIQGVQIQESTSWNEYTFFNLTAGSKYNIAITAVSGGKRSFSVYTNGSTVPSPVKDIGISTKANSLLISWSHGSGNVERYRLMLMDKGILVHGGVVDKHATSYAFHGLTPGYLYNLTVMTEAAGLQNYRWKLVRTAPMEVSNLKVTNDGSLTSLKVKWQRPPGNVDSYNITLSHKGTIKESRVLAPWITETHFKELVPGRLYQVTVSCVSGELSAQKMAVGRTFPDKVANLEANNNGRMRSLVVSWSPPAGDWEQYRILLFNDSVVLLNITVGKEETQYVMDDTGLVPGRQYEVEVIVESGNLKNSERCQGRTVPLAVLQLRVKHANETSLSIMWQTPVAEWEKYIISLADRDLLLIHKSLSKDAKEFTFTDLVPGRKYMATVTSISGDLKNSSSVKGRTVPAQVTDLHVANQGMTSSLFTNWTQAQGDVEFYQVLLIHENVVIKNESISSETSRYSFHSLKSGSLYSVVVTTVSGGISSRQVVVEGRTVPSSVSGVTVNNSGRNDYLSVSWLLAPGDVDNYEVTLSHDGKVVQSLVIAKSVRECSFSSLTPGRLYTVTITTRSGKYENHSFSQERTVPDKVQGVSVSNSARSDYLRVSWVHATGDFDHYEVTIKNKNNFIQTKSIPKSENECVFVQLVPGRLYSVTVTTKSGQYEANEQGNGRTIPEPVKDLTLRNRSTEDLHVTWSGANGDVDQYEIQLLFNDMKVFPPFHLVNTATEYRFTSLTPGRQYKILVLTISGDVQQSAFIEGFTVPSAVKNIHISPNGATDSLTVNWTPGGGDVDSYTVSAFRHSQKVDSQTIPKHVFEHTFHRLEAGEQYQIMIASVSGSLKNQINVVGRTVPASVQGVIADNAYSSYSLIVSWQKAAGVAERYDILLLTENGILLRNTSEPATTKQHKFEDLTPGKKYKIQILTVSGGLFSKEAQTEGRTVPAAVTDLRITENSTRHLSFRWTASEGELSWYNIFLYNPDGNLQERAQVDPLVQSFSFQNLLQGRMYKMVIVTHSGELSNESFIFGRTVPASVSHLRGSNRNTTDSLWFNWSPASGDFDFYELILYNPNGTKKENWKDKDLTEWRFQGLVPGRKYVLWVVTHSGDLSNKVTAESRTAPSPPSLMSFADIANTSLAITWKGPPDWTDYNDFELQWLPRDALTVFNPYNNRKSEGRIVYGLRPGRSYQFNVKTVSGDSWKTYSKPIFGSVRTKPDKIQNLHCRPQNSTAIACSWIPPDSDFDGYSIECRKMDTQEVEFSRKLEKEKSLLNIMMLVPHKRYLVSIKVQSAGMTSEVVEDSTITMIDRPPPPPPHIRVNEKDVLISKSSINFTVNCSWFSDTNGAVKYFTVVVREADGSDELKPEQQHPLPSYLEYRHNASIRVYQTNYFASKCAENPNSNSKSFNIKLGAEMESLGGKCDPTQQKFCDGPLKPHTAYRISIRAFTQLFDEDLKEFTKPLYSDTFFSLPITTESEPLFGAIEGVSAGLFLIGMLVAVVALLICRQKVSHGRERPSARLSIRRDRPLSVHLNLGQKGNRKTSCPIKINQFEGHFMKLQADSNYLLSKEYEELKDVGRNQSCDIALLPENRGKNRYNNILPYDATRVKLSNVDDDPCSDYINASYIPGNNFRREYIVTQGPLPGTKDDFWKMVWEQNVHNIVMVTQCVEKGRVKCDHYWPADQDSLYYGDLILQMLSESVLPEWTIREFKICGEEQLDAHRLIRHFHYTVWPDHGVPETTQSLIQFVRTVRDYINRSPGAGPTVVHCSAGVGRTGTFIALDRILQQLDSKDSVDIYGAVHDLRLHRVHMVQTECQYVYLHQCVRDVLRARKLRSEQENPLFPIYENVNPEYHRDPVYSRH",
# PTPRH_HUMAN
"MAGAGGGLGVWGNLVLLGLCSWTGARAPAPNPGRNLTVETQTTSSISLSWEVPDGLDSQNSNYWVQCTGDGGTTETRNTTATNVTVDGLGPGSLYTCSVWVEKDGVNSSVGTVTTATAPNPVRNLRVEAQTNSSIALTWEVPDGPDPQNSTYGVEYTGDGGRAGTRSTAHTNITVDGLEPGCLYAFSMWVGKNGINSSRETRNATTAHNPVRNLRVEAQTTSSISLSWEVPDGTDPQNSTYCVQCTGDGGRTETRNTTDTRVTVDGLGPGSLYTCSVWVEKDGVNSSVEIVTSATAPNPVRNLTVEAQTNSSIALTWEVPDGPDPQNSTYGVEYTGDGGRAGTRSTAHTNITVDRLEPGCLYVFSVWVGKNGINSSRETRNATTAPNPVRNLHMETQTNSSIALCWEVPDGPYPQDYTYWVEYTGDGGGTETRNTTNTSVTAERLEPGTLYTFSVWAEKNGARGSRQNVSISTVPNAVTSLSKQDWTNSTIALRWTAPQGPGQSSYSYWVSWVREGMTDPRTQSTSGTDITLKELEAGSLYHLTVWAERNEVRGYNSTLTAATAPNEVTDLQNETQTKNSVMLWWKAPGDPHSQLYVYWVQWASKGHPRRGQDPQANWVNQTSRTNETWYKVEALEPGTLYNFTVWAERNDVASSTQSLCASTYPDTVTITSCVSTSAGYGVNLIWSCPQGGYEAFELEVGGQRGSQDRSSCGEAVSVLGLGPARSYPATITTIWDGMKVVSHSVVCHTESAGVIAGAFVGILLFLILVGLLIFFLKRRNKKKQQKPELRDLVFSSPGDIPAEDFADHVRKNERDSNCGFADEYQQLSLVGHSQSQMVASASENNAKNRYRNVLPYDWSRVPLKPIHEEPGSDYINASFMPGLWSPQEFIATQGPLPQTVGDFWRLVWEQQSHTLVMLTNCMEAGRVKCEHYWPLDSQPCTHGHLRVTLVGEEVMENWTVRELLLLQVEEQKTLSVRQFHYQAWPDHGVPSSPDTLLAFWRMLRQWLDQTMEGGPPIVHCSAGVGRTGTLIALDVLLRQLQSEGLLGPFSFVRKMRESRPLMVQTEAQYVFLHQCILRFLQQSAQAPAEKEVPYEDVENLIYENVAAIQAHKLEV",
# PTPRJ_HUMAN
"MKPAAREARLPPRSPGLRWALPLLLLLLRLGQILCAGGTPSPIPDPSVATVATGENGITQISSTAESFHKQNGTGTPQVETNTSEDGESSGANDSLRTPEQGSNGTDGASQKTPSSTGPSPVFDIKAVSISPTNVILTWKSNDTAASEYKYVVKHKMENEKTITVVHQPWCNITGLRPATSYVFSITPGIGNETWGDPRVIKVITEPIPVSDLRVALTGVRKAALSWSNGNGTASCRVLLESIGSHEELTQDSRLQVNISGLKPGVQYNINPYLLQSNKTKGDPLGTEGGLDASNTERSRAGSPTAPVHDESLVGPVDPSSGQQSRDTEVLLVGLEPGTRYNATVYSQAANGTEGQPQAIEFRTNAIQVFDVTAVNISATSLTLIWKVSDNESSSNYTYKIHVAGETDSSNLNVSEPRAVIPGLRSSTFYNITVCPVLGDIEGTPGFLQVHTPPVPVSDFRVTVVSTTEIGLAWSSHDAESFQMHITQEGAGNSRVEITTNQSIIIGGLFPGTKYCFEIVPKGPNGTEGASRTVCNRTVPSAVFDIHVVYVTTTEMWLDWKSPDGASEYVYHLVIESKHGSNHTSTYDKAITLQGLIPGTLYNITISPEVDHVWGDPNSTAQYTRPSNVSNIDVSTNTTAATLSWQNFDDASPTYSYCLLIEKAGNSSNATQVVTDIGITDATVTELIPGSSYTVEIFAQVGDGIKSLEPGRKSFCTDPASMASFDCEVVPKEPALVLKWTCPPGANAGFELEVSSGAWNNATHLESCSSENGTEYRTEVTYLNFSTSYNISITTVSCGKMAAPTRNTCTTGITDPPPPDGSPNITSVSHNSVKVKFSGFEASHGPIKAYAVILTTGEAGHPSADVLKYTYEDFKKGASDTYVTYLIRTEEKGRSQSLSEVLKYEIDVGNESTTLGYYNGKLEPLGSYRACVAGFTNITFHPQNKGLIDGAESYVSFSRYSDAVSLPQDPGVICGAVFGCIFGALVIVTVGGFIFWRKKRKDAKNNEVSFSQIKPKKSKLIRVENFEAYFKKQQADSNCGFAEEYEDLKLVGISQPKYAAELAENRGKNRYNNVLPYDISRVKLSVQTHSTDDYINANYMPGYHSKKDFIATQGPLPNTLKDFWRMVWEKNVYAIIMLTKCVEQGRTKCEEYWPSKQAQDYGDITVAMTSEIVLPEWTIRDFTVKNIQTSESHPLRQFHFTSWPDHGVPDTTDLLINFRYLVRDYMKQSPPESPILVHCSAGVGRTGTFIAIDRLIYQIENENTVDVYGIVYDLRMHRPLMVQTEDQYVFLNQCVLDIVRSQKDSKVDLIYQNTTAMTIYENLAPVTTFGKTNGYIA",
# PTPRO_HUMAN
"MGHLPTGIHGARRLLPLLWLFVLFKNATAFHVTVQDDNNIVVSLEASDVISPASVYVVKITGESKNYFFEFEEFNSTLPPPVIFKASYHGLYYIITLVVVNGNVVTKPSRSITVLTKPLPVTSVSIYDYKPSPETGVLFEIHYPEKYNVFTRVNISYWEGKDFRTMLYKDFFKGKTVFNHWLPGMCYSNITFQLVSEATFNKSTLVEYSGVSHEPKQHRTAPYPPQNISVRIVNLNKNNWEEQSGNFPEESFMRSQDTIGKEKLFHFTEETPEIPSGNISSGWPDFNSSDYETTSQPYWWDSASAAPESEDEFVSVLPMEYENNSTLSETEKSTSGSFSFFPVQMILTWLPPKPPTAFDGFHIHIEREENFTEYLMVDEEAHEFVAELKEPGKYKLSVTTFSSSGSCETRKSQSAKSLSFYISPSGEWIEELTEKPQHVSVHVLSSTTALMSWTSSQENYNSTIVSVVSLTCQKQKESQRLEKQYCTQVNSSKPIIENLVPGAQYQVVIYLRKGPLIGPPSDPVTFAIVPTGIKDLMLYPLGPTAVVLSWTRPYLGVFRKYVVEMFYFNPATMTSEWTTYYEIAATVSLTASVRIANLLPAWYYNFRVTMVTWGDPELSCCDSSTISFITAPVAPEITSVEYFNSLLYISWTYGDDTTDLSHSRMLHWMVVAEGKKKIKKSVTRNVMTAILSLPPGDIYNLSVTACTERGSNTSMLRLVKLEPAPPKSLFAVNKTQTSVTLLWVEEGVADFFEVFCQQVGSSQKTKLQEPVAVSSHVVTISSLLPATAYNCSVTSFSHDSPSVPTFIAVSTMVTEMNPNVVVISVLAILSTLLIGLLLVTLIILRKKHLQMARECGAGTFVNFASLERDGKLPYNWRRSIFAFLTLLPSCLWTDYLLAFYINPWSKNGLKKRKLTNPVQLDDFDAYIKDMAKDSDYKFSLQFEELKLIGLDIPHFAADLPLNRCKNRYTNILPYDFSRVRLVSMNEEEGADYINANYIPGYNSPQEYIATQGPLPETRNDFWKMVLQQKSQIIVMLTQCNEKRRVKCDHYWPFTEEPIAYGDITVEMISEEEQDDWACRHFRINYADEMQDVMHFNYTAWPDHGVPTANAAESILQFVHMVRQQATKSKGPMIIHCSAGVGRTGTFIALDRLLQHIRDHEFVDILGLVSEMRSYRMSMVQTEEQYIFIHQCVQLMWMKKKQQFCISDVIYENVSKS",
# PTPRU_HUMAN
"MARAQALVLALTFQLCAPETETPAAGCTFEEASDPAVPCEYSQAQYDDFQWEQVRIHPGTRAPADLPHGSYLMVNTSQHAPGQRAHVIFQSLSENDTHCVQFSYFLYSRDGHSPGTLGVYVRVNGGPLGSAVWNMTGSHGRQWHQAELAVSTFWPNEYQVLFEALISPDRRGYMGLDDILLLSYPCAKAPHFSRLGDVEVNAGQNASFQCMAAGRAAEAERFLLQRQSGALVPAAGVRHISHRRFLATFPLAAVSRAEQDLYRCVSQAPRGAGVSNFAELIVKEPPTPIAPPQLLRAGPTYLIIQLNTNSIIGDGPIVRKEIEYRMARGPWAEVHAVSLQTYKLWHLDPDTEYEISVLLTRPGDGGTGRPGPPLISRTKCAEPMRAPKGLAFAEIQARQLTLQWEPLGYNVTRCHTYTVSLCYHYTLGSSHNQTIRECVKTEQGVSRYTIKNLLPYRNVHVRLVLTNPEGRKEGKEVTFQTDEDVPSGIAAESLTFTPLEDMIFLKWEEPQEPNGLITQYEISYQSIESSDPAVNVPGPRRTISKLRNETYHVFSNLHPGTTYLFSVRARTGKGFGQAALTEITTNISAPSFDYADMPSPLGESENTITVLLRPAQGRGAPISVYQVIVEEERARRLRREPGGQDCFPVPLTFEAALARGLVHYFGAELAASSLPEAMPFTVGDNQTYRGFWNPPLEPRKAYLIYFQAASHLKGETRLNCIRIARKAACKESKRPLEVSQRSEEMGLILGICAGGLAVLILLLGAIIVIIRKGRDHYAYSYYPKPVNMTKATVNYRQEKTHMMSAVDRSFTDQSTLQEDERLGLSFMDTHGYSTRGDQRSGGVTEASSLLGGSPRRPCGRKGSPYHTGQLHPAVRVADLLQHINQMKTAEGYGFKQEYESFFEGWDATKKKDKVKGSRQEPMPAYDRHRVKLHPMLGDPNADYINANYIDGYHRSNHFIATQGPKPEMVYDFWRMVWQEHCSSIVMITKLVEVGRVKCSRYWPEDSDTYGDIKIMLVKTETLAEYVVRTFALERRGYSARHEVRQFHFTAWPEHGVPYHATGLLAFIRRVKASTPPDAGPIVIHCSAGTGRTGCYIVLDVMLDMAECEGVVDIYNCVKTLCSRRVNMIQTEEQYIFIHDAILEACLCGETTIPVSEFKATYKEMIRIDPQSNSSQLREEFQTLNSVTPPLDVEECSIALLPRNRDKNRSMDVLPPDRCLPFLISTDGDSNNYINAALTDSYTRSAAFIVTLHPLQSTTPDFWRLVYDYGCTSIVMLNQLNQSNSAWPCLQYWPEPGRQQYGLMEVEFMSGTADEDLVARVFRVQNISRLQEGHLLVRHFQFLRWSAYRDTPDSKKAFLHLLAEVDKWQAESGDGRTIVHCLNGGGRSGTFCACATVLEMIRCHNLVDVFFAAKTLRNYKPNMVETMDQYHFCYDVALEYLEGLESR",
# PTPRQ_HUMAN
"MKKVPIKPEQPEKLRAFNISTHSFSLHWSLPSGHVERYQVDLVPDSGFVTIRDLGGGEYQVDVSNVVPGTRYDITISSISTTYTSPVTRIVTTNVTKPGPPVFLAGERVGSAGILLSWNTPPNPNGRIISYIVKYKEVCPWMQTVYTQVRSKPDSLEVLLTNLNPGTTYEIKVAAENSAGIGVFSDPFLFQTAESAPGKVVNLTVEAYNASAVKLIWYLPRQPNGKITSFKISVKHARSGIVVKDVSIRVEDILTGKLPECNENSESFLWSTASPSPTLGRVTPPSRTTHSSSTLTQNEISSVWKEPISFVVTHLRPYTTYLFEVSAVTTEAGYIDSTIVRTPESVPEGPPQNCVTGNITGKSFSILWDPPTIVTGKFSYRVELYGPSGRILDNSTKDLKFAFTNLTPFTMYDVYIAAETSAGTGPKSNISVFTPPDVPGAVFDLQLAEVESTQVRITWKKPRQPNGIINQYRVKVLVPETGIILENTLLTGNNEYINDPMAPEIVNIVEPMVGLYEGSAEMSSDLHSLATFIYNSHPDKNFPARNRAEDQTSPVVTTRNQYITDIAAEQLSYVIRRLVPFTEHMISVSAFTIMGEGPPTVLSVRTRQQVPSSIKIINYKNISSSSILLYWDPPEYPNGKITHYTIYAMELDTNRAFQITTIDNSFLITGLKKYTKYKMRVAASTHVGESSLSEENDIFVRTSEDEPESSPQDVEVIDVTADEIRLKWSPPEKPNGIIIAYEVLYKNIDTLYMKNTSTTDIILRNLRPHTLYNISVRSYTRFGHGNQVSSLLSVRTSETVPDSAPENITYKNISSGEIELSFLPPSSPNGIIKKYTIYLKRSNGNEERTINTTSLTQNIKVLKKYTQYIIEVSASTLKGEGVRSAPISILTEEDAPDSPPQDFSVKQLSGVTVKLSWQPPLEPNGIILYYTVYVWNRSSLKTINVTETSLELSDLDYNVEYSAYVTASTRFGDGKTRSNIISFQTPEGAPSDPPKDVYYANLSSSSIILFWTPPSKPNGIIQYYSVYYRNTSGTFMQNFTLHEVTNDFDNMTVSTIIDKLTIFSYYTFWLTASTSVGNGNKSSDIIEVYTDQDIPEGFVGNLTYESISSTAINVSWVPPAQPNGLVFYYVSLILQQTPRHVRPPLVTYERSIYFDNLEKYTDYILKITPSTEKGFSDTYTAQLYIKTEEDVPETSPIINTFKNLSSTSVLLSWDPPVKPNGAIISYDLTLQGPNENYSFITSDNYIILEELSPFTLYSFFAAARTRKGLGPSSILFFYTDESVPLAPPQNLTLINCTSDFVWLKWSPSPLPGGIVKVYSFKIHEHETDTIYYKNISGFKTEAKLVGLEPVSTYSIRVSAFTKVGNGNQFSNVVKFTTQESVPDVVQNMQCMATSWQSVLVKWDPPKKANGIITQYMVTVERNSTKVSPQDHMYTFIKLLANTSYVFKVRASTSAGEGDESTCHVSTLPETVPSVPTNIAFSDVQSTSATLTWIRPDTILGYFQNYKITTQLRAQKCKEWESEECVEYQKIQYLYEAHLTEETVYGLKKFRWYRFQVAASTNAGYGNASNWISTKTLPGPPDGPPENVHVVATSPFSISISWSEPAVITGPTCYLIDVKSVDNDEFNISFIKSNEENKTIEIKDLEIFTRYSVVITAFTGNISAAYVEGKSSAEMIVTTLESAPKDPPNNMTFQKIPDEVTKFQLTFLPPSQPNGNIQVYQALVYREDDPTAVQIHNLSIIQKTNTFVIAMLEGLKGGHTYNISVYAVNSAGAGPKVPMRITMDIKAPARPKTKPTPIYDATGKLLVTSTTITIRMPICYYSDDHGPIKNVQVLVTETGAQHDGNVTKWYDAYFNKARPYFTNEGFPNPPCTEGKTKFSGNEEIYIIGADNACMIPGNEDKICNGPLKPKKQYLFKFRATNIMGQFTDSDYSDPVKTLGEGLSERTVEIILSVTLCILSIILLGTAIFAFARIRQKQKEGGTYSPQDAEIIDTKLKLDQLITVADLELKDERLTRPISKKSFLQHVEELCTNNNLKFQEEFSELPKFLQDLSSTDADLPWNRAKNRFPNIKPYNNNRVKLIADASVPGSDYINASYISGYLCPNEFIATQGPLPGTVGDFWRMVWETRAKTLVMLTQCFEKGRIRCHQYWPEDNKPVTVFGDIVITKLMEDVQIDWTIRDLKIERHGDCMTVRQCNFTAWPEHGVPENSAPLIHFVKLVRASRAHDTTPMIVHCSAGVGRTGVFIALDHLTQHINDHDFVDIYGLVAELRSERMCMVQNLAQYIFLHQCILDLLSNKGSNQPICFVNYSALQKMDSLDAMEGDVELEWEETTM",
# PTPRC_HUMAN
"MTMYLWLKLLAFGFAFLDTEVFVTGQSPTPSPTGLTTAKMPSVPLSSDPLPTHTTAFSPASTFERENDFSETTTSLSPDNTSTQVSPDSLDNASAFNTTGVSSVQTPHLPTHADSQTPSAGTDTQTFSGSAANAKLNPTPGSNAISDVPGERSTASTFPTDPVSPLTTTLSLAHHSSAALPARTSNTTITANTSDAYLNASETTTLSPSGSAVISTTTIATTPSKPTCDEKYANITVDYLYNKETKLFTAKLNVNENVECGNNTCTNNEVHNLTECKNASVSISHNSCTAPDKTLILDVPPGVEKFQLHDCTQVEKADTTICLKWKNIETFTCDTQNITYRFQCGNMIFDNKEIKLENLEPEHEYKCDSEILYNNHKFTNASKIIKTDFGSPGEPQIIFCRSEAAHQGVITWNPPQRSFHNFTLCYIKETEKDCLNLDKNLIKYDLQNLKPYTKYVLSLHAYIIAKVQRNGSAAMCHFTTKSAPPSQVWNMTVSMTSDNSMHVKCRPPRDRNGPHERYHLEVEAGNTLVRNESHKNCDFRVKDLQYSTDYTFKAYFHNGDYPGEPFILHHSTSYNSKALIAFLAFLIIVTSIALLVVLYKIYDLHKKRSCNLDEQQELVERDDEKQLMNVEPIHADILLETYKRKIADEGRLFLAEFQSIPRVFSKFPIKEARKPFNQNKNRYVDILPYDYNRVELSEINGDAGSNYINASYIDGFKEPRKYIAAQGPRDETVDDFWRMIWEQKATVIVMVTRCEEGNRNKCAEYWPSMEEGTRAFGDVVVKINQHKRCPDYIIQKLNIVNKKEKATGREVTHIQFTSWPDHGVPEDPHLLLKLRRRVNAFSNFFSGPIVVHCSAGVGRTGTYIGIDAMLEGLEAENKVDVYGYVVKLRRQRCLMVQVEAQYILIHQALVEYNQFGETEVNLSELHPYLHNMKKRDPPSEPSPLEAEFQRLPSYRSWRTQHIGNQEENKSKNRNSNVIPYDYNRVPLKHELEMSKESEHDSDESSDDDSDSEEPSKYINASFIMSYWKPEVMIAAQGPLKETIGDFWQMIFQRKVKVIVMLTELKHGDQEICAQYWGEGKQTYGDIEVDLKDTDKSSTYTLRVFELRHSKRKDSRTVYQYQYTNWSVEQLPAEPKELISMIQVVKQKLPQKNSSEGNKHHKSTPLLIHCRDGSQQTGIFCALLNLLESAETEEVVDIFQVVKALRKARPGMVSTFEQYQFLYDVIASTYPAQNGQVKKNNHQEDKIEFDNEVDKVKQDANCVNPLGAPEKLPEAKEQAEGSEPTSGTEGPEHSVNGPASPALNQGS",
# PTPRD_HUMAN
"MVHVARLLLLLLTFFLRTDAETPPRFTRTPVDQTGVSGGVASFICQATGDPRPKIVWNKKGKKVSNQRFEVIEFDDGSGSVLRIQPLRTPRDEAIYECVASNNVGEISVSTRLTVLREDQIPRGFPTIDMGPQLKVVERTRTATMLCAASGNPDPEITWFKDFLPVDTSNNNGRIKQLRSESIGGTPIRGALQIEQSEESDQGKYECVATNSAGTRYSAPANLYVRELREVRRVPPRFSIPPTNHEIMPGGSVNITCVAVGSPMPYVKWMLGAEDLTPEDDMPIGRNVLELNDVRQSANYTCVAMSTLGVIEAIAQITVKALPKPPGTPVVTESTATSITLTWDSGNPEPVSYYIIQHKPKNSEELYKEIDGVATTRYSVAGLSPYSDYEFRVVAVNNIGRGPPSEPVLTQTSEQAPSSAPRDVQARMLSSTTILVQWKEPEEPNGQIQGYRVYYTMDPTQHVNNWMKHNVADSQITTIGNLVPQKTYSVKVLAFTSIGDGPLSSDIQVITQTGVPGQPLNFKAEPESETSILLSWTPPRSDTIANYELVYKDGEHGEEQRITIEPGTSYRLQGLKPNSLYYFRLAARSPQGLGASTAEISARTMQSKPSAPPQDISCTSPSSTSILVSWQPPPVEKQNGIITEYSIKYTAVDGEDDKPHEILGIPSDTTKYLLEQLEKWTEYRITVTAHTDVGPGPESLSVLIRTNEDVPSGPPRKVEVEAVNSTSVKVSWRSPVPNKQHGQIRGYQVHYVRMENGEPKGQPMLKDVMLADAQWEFDDTTEHDMIISGLQPETSYSLTVTAYTTKGDGARSKPKLVSTTGAVPGKPRLVINHTQMNTALIQWHPPVDTFGPLQGYRLKFGRKDMEPLTTLEFSEKEDHFTATDIHKGASYVFRLSARNKVGFGEEMVKEISIPEEVPTGFPQNLHSEGTTSTSVQLSWQPPVLAERNGIITKYTLLYRDINIPLLPMEQLIVPADTTMTLTGLKPDTTYDVKVRAHTSKGPGPYSPSVQFRTLPVDQVFAKNFHVKAVMKTSVLLSWEIPENYNSAMPFKILYDDGKMVEEVDGRATQKLIVNLKPEKSYSFVLTNRGNSAGGLQHRVTAKTAPDVLRTKPAFIGKTNLDGMITVQLPEVPANENIKGYYIIIVPLKKSRGKFIKPWESPDEMELDELLKEISRKRRSIRYGREVELKPYIAAHFDVLPTEFTLGDDKHYGGFTNKQLQSGQEYVFFVLAVMEHAESKMYATSPYSDPVVSMDLDPQPITDEEEGLIWVVGPVLAVVFIICIVIAILLYKRKRAESDSRKSSIPNNKEIPSHHPTDPVELRRLNFQTPGMASHPPIPILELADHIERLKANDNLKFSQEYESIDPGQQFTWEHSNLEVNKPKNRYANVIAYDHSRVLLSAIEGIPGSDYVNANYIDGYRKQNAYIATQGSLPETFGDFWRMIWEQRSATVVMMTKLEERSRVKCDQYWPSRGTETHGLVQVTLLDTVELATYCVRTFALYKNGSSEKREVRQFQFTAWPDHGVPEHPTPFLAFLRRVKTCNPPDAGPMVVHCSAGVGRTGCFIVIDAMLERIKHEKTVDIYGHVTLMRAQRNYMVQTEDQYIFIHDALLEAVTCGNTEVPARNLYAYIQKLTQIETGENVTGMELEFKRLASSKAHTSRFISANLPCNKFKNRLVNIMPYESTRVCLQPIRGVEGSDYINASFIDGYRQQKAYIATQGPLAETTEDFWRMLWEHNSTIVVMLTKLREMGREKCHQYWPAERSARYQYFVVDPMAEYNMPQYILREFKVTDARDGQSRTVRQFQFTDWPEQGVPKSGEGFIDFIGQVHKTKEQFGQDGPISVHCSAGVGRTGVFITLSIVLERMRYEGVVDIFQTVKMLRTQRPAMVQTEDQYQFSYRAALEYLGSFDHYAT",
# PTPRF_HUMAN
"MAPEPAPGRTMVPLVPALVMLGLVAGAHGDSKPVFIKVPEDQTGLSGGVASFVCQATGEPKPRITWMKKGKKVSSQRFEVIEFDDGAGSVLRIQPLRVQRDEAIYECTATNSLGEINTSAKLSVLEEEQLPPGFPSIDMGPQLKVVEKARTATMLCAAGGNPDPEISWFKDFLPVDPATSNGRIKQLRSGALQIESSEESDQGKYECVATNSAGTRYSAPANLYVRVRRVAPRFSIPPSSQEVMPGGSVNLTCVAVGAPMPYVKWMMGAEELTKEDEMPVGRNVLELSNVVRSANYTCVAISSLGMIEATAQVTVKALPKPPIDLVVTETTATSVTLTWDSGNSEPVTYYGIQYRAAGTEGPFQEVDGVATTRYSIGGLSPFSEYAFRVLAVNSIGRGPPSEAVRARTGEQAPSSPPRRVQARMLSASTMLVQWEPPEEPNGLVRGYRVYYTPDSRRPPNAWHKHNTDAGLLTTVGSLLPGITYSLRVLAFTAVGDGPPSPTIQVKTQQGVPAQPADFQAEVESDTRIQLSWLLPPQERIIMYELVYWAAEDEDQQHKVTFDPTSSYTLEDLKPDTLYRFQLAARSDMGVGVFTPTIEARTAQSTPSAPPQKVMCVSMGSTTVRVSWVPPPADSRNGVITQYSVAYEAVDGEDRGRHVVDGISREHSSWDLVGLEKWTEYRVWVRAHTDVGPGPESSPVLVRTDEDVPSGPPRKVEVEPLNSTAVHVYWKLPVPSKQHGQIRGYQVTYVRLENGEPRGLPIIQDVMLAEAQWRPEESEDYETTISGLTPETTYSVTVAAYTTKGDGARSKPKIVTTTGAVPGRPTMMISTTAMNTALLQWHPPKELPGELLGYRLQYCRADEARPNTIDFGKDDQHFTVTGLHKGTTYIFRLAAKNRAGLGEEFEKEIRTPEDLPSGFPQNLHVTGLTTSTTELAWDPPVLAERNGRIISYTVVFRDINSQQELQNITTDTRFTLTGLKPDTTYDIKVRAWTSKGSGPLSPSIQSRTMPVEQVFAKNFRVAAAMKTSVLLSWEVPDSYKSAVPFKILYNGQSVEVDGHSMRKLIADLQPNTEYSFVLMNRGSSAGGLQHLVSIRTAPDLLPHKPLPASAYIEDGRFDLSMPHVQDPSLVRWFYIVVVPIDRVGGSMLTPRWSTPEELELDELLEAIEQGGEEQRRRRRQAERLKPYVAAQLDVLPETFTLGDKKNYRGFYNRPLSPDLSYQCFVLASLKEPMDQKRYASSPYSDEIVVQVTPAQQQEEPEMLWVTGPVLAVILIILIVIAILLFKRKRTHSPSSKDEQSIGLKDSLLAHSSDPVEMRRLNYQTPGMRDHPPIPITDLADNIERLKANDGLKFSQEYESIDPGQQFTWENSNLEVNKPKNRYANVIAYDHSRVILTSIDGVPGSDYINANYIDGYRKQNAYIATQGPLPETMGDFWRMVWEQRTATVVMMTRLEEKSRVKCDQYWPARGTETCGLIQVTLLDTVELATYTVRTFALHKSGSSEKRELRQFQFMAWPDHGVPEYPTPILAFLRRVKACNPLDAGPMVVHCSAGVGRTGCFIVIDAMLERMKHEKTVDIYGHVTCMRSQRNYMVQTEDQYVFIHEALLEAATCGHTEVPARNLYAHIQKLGQVPPGESVTAMELEFKLLASSKAHTSRFISANLPCNKFKNRLVNIMPYELTRVCLQPIRGVEGSDYINASFLDGYRQQKAYIATQGPLAESTEDFWRMLWEHNSTIIVMLTKLREMGREKCHQYWPAERSARYQYFVVDPMAEYNMPQYILREFKVTDARDGQSRTIRQFQFTDWPEQGVPKTGEGFIDFIGQVHKTKEQFGQDGPITVHCSAGVGRTGVFITLSIVLERMRYEGVVDMFQTVKTLRTQRPAMVQTEDQYQLCYRAALEYLGSFDHYAT",
# PTPRS_HUMAN
"MAPTWGPGMVSVVGPMGLLVVLLVGGCAAEEPPRFIKEPKDQIGVSGGVASFVCQATGDPKPRVTWNKKGKKVNSQRFETIEFDESAGAVLRIQPLRTPRDENVYECVAQNSVGEITVHAKLTVLREDQLPSGFPNIDMGPQLKVVERTRTATMLCAASGNPDPEITWFKDFLPVDPSASNGRIKQLRSETFESTPIRGALQIESSEETDQGKYECVATNSAGVRYSSPANLYVRELREVRRVAPRFSILPMSHEIMPGGNVNITCVAVGSPMPYVKWMQGAEDLTPEDDMPVGRNVLELTDVKDSANYTCVAMSSLGVIEAVAQITVKSLPKAPGTPMVTENTATSITITWDSGNPDPVSYYVIEYKSKSQDGPYQIKEDITTTRYSIGGLSPNSEYEIWVSAVNSIGQGPPSESVVTRTGEQAPASAPRNVQARMLSATTMIVQWEEPVEPNGLIRGYRVYYTMEPEHPVGNWQKHNVDDSLLTTVGSLLEDETYTVRVLAFTSVGDGPLSDPIQVKTQQGVPGQPMNLRAEARSETSITLSWSPPRQESIIKYELLFREGDHGREVGRTFDPTTSYVVEDLKPNTEYAFRLAARSPQGLGAFTPVVRQRTLQSKPSAPPQDVKCVSVRSTAILVSWRPPPPETHNGALVGYSVRYRPLGSEDPEPKEVNGIPPTTTQILLEALEKWTQYRITTVAHTEVGPGPESSPVVVRTDEDVPSAPPRKVEAEALNATAIRVLWRSPAPGRQHGQIRGYQVHYVRMEGAEARGPPRIKDVMLADAQWETDDTAEYEMVITNLQPETAYSITVAAYTMKGDGARSKPKVVVTKGAVLGRPTLSVQQTPEGSLLARWEPPAGTAEDQVLGYRLQFGREDSTPLATLEFPPSEDRYTASGVHKGATYVFRLAARSRGGLGEEAAEVLSIPEDTPRGHPQILEAAGNASAGTVLLRWLPPVPAERNGAIVKYTVAVREAGALGPARETELPAAAEPGAENALTLQGLKPDTAYDLQVRAHTRRGPGPFSPPVRYRTFLRDQVSPKNFKVKMIMKTSVLLSWEFPDNYNSPTPYKIQYNGLTLDVDGRTTKKLITHLKPHTFYNFVLTNRGSSLGGLQQTVTAWTAFNLLNGKPSVAPKPDADGFIMVYLPDGQSPVPVQSYFIVMVPLRKSRGGQFLTPLGSPEDMDLEELIQDISRLQRRSLRHSRQLEVPRPYIAARFSVLPPTFHPGDQKQYGGFDNRGLEPGHRYVLFVLAVLQKSEPTFAASPFSDPFQLDNPDPQPIVDGEEGLIWVIGPVLAVVFIICIVIAILLYKNKPDSKRKDSEPRTKCLLNNADLAPHHPKDPVEMRRINFQTPDSGLRSPLREPGFHFESMLSHPPIPIADMAEHTERLKANDSLKLSQEYESIDPGQQFTWEHSNLEVNKPKNRYANVIAYDHSRVILQPIEGIMGSDYINANYVDGYRCQNAYIATQGPLPETFGDFWRMVWEQRSATIVMMTRLEEKSRIKCDQYWPNRGTETYGFIQVTLLDTIELATFCVRTFSLHKNGSSEKREVRQFQFTAWPDHGVPEYPTPFLAFLRRVKTCNPPDAGPIVVHCSAGVGRTGCFIVIDAMLERIKPEKTVDVYGHVTLMRSQRNYMVQTEDQYSFIHEALLEAVGCGNTEVPARSLYAYIQKLAQVEPGEHVTGMELEFKRLANSKAHTSRFISANLPCNKFKNRLVNIMPYESTRVCLQPIRGVEGSDYINASFIDGYRQQKAYIATQGPLAETTEDFWRMLWENNSTIVVMLTKLREMGREKCHQYWPAERSARYQYFVVDPMAEYNMPQYILREFKVTDARDGQSRTVRQFQFTDWPEQGVPKSGEGFIDFIGQVHKTKEQFGQDGPISVHCSAGVGRTGVFITLSIVLERMRYEGVVDIFQTVKMLRTQRPAMVQTEDEYQFCYQAALEYLGSFDHYAT",
# PTPRG_HUMAN
"MRRLLEPCWWILFLKITSSVLHYVVCFPALTEGYVGALHENRHGSAVQIRRRKASGDPYWAYSGAYGPEHWVTSSVSCGGRHQSPIDILDQYARVGEEYQELQLDGFDNESSNKTWMKNTGKTVAILLKDDYFVSGAGLPGRFKAEKVEFHWGHSNGSAGSEHSINGRRFPVEMQIFFYNPDDFDSFQTAISENRIIGAMAIFFQVSPRDNSALDPIIHGLKGVVHHEKETFLDPFVLRDLLPASLGSYYRYTGSLTTPPCSEIVEWIVFRRPVPISYHQLEAFYSIFTTEQQDHVKSVEYLRNNFRPQQRLHDRVVSKSAVRDSWNHDMTDFLENPLGTEASKVCSSPPIHMKVQPLNQTALQVSWSQPETIYHPPIMNYMISYSWTKNEDEKEKTFTKDSDKDLKATISHVSPDSLYLFRVQAVCRNDMRSDFSQTMLFQANTTRIFQGTRIVKTGVPTASPASSADMAPISSGSSTWTSSGIPFSFVSMATGMGPSSSGSQATVASVVTSTLLAGLGFGGGGISSFPSTVWPTRLPTAASASKQAARPVLATTEALASPGPDGDSSPTKDGEGTEEGEKDEKSESEDGEREHEEDGEKDSEKKEKSGVTHAAEERNQTEPSPTPSSPNRTAEGGHQTIPGHEQDHTAVPTDQTGGRRDAGPGLDPDMVTSTQVPPTATEEQYAGSDPKRPEMPSKKPMSRGDRFSEDSRFITVNPAEKNTSGMISRPAPGRMEWIIPLIVVSALTFVCLILLIAVLVYWRGCNKIKSKGFPRRFREVPSSGERGEKGSRKCFQTAHFYVEDSSSPRVVPNESIPIIPIPDDMEAIPVKQFVKHIGELYSNNQHGFSEDFEEVQRCTADMNITAEHSNHPENKHKNRYINILAYDHSRVKLRPLPGKDSKHSDYINANYVDGYNKAKAYIATQGPLKSTFEDFWRMIWEQNTGIIVMITNLVEKGRRKCDQYWPTENSEEYGNIIVTLKSTKIHACYTVRRFSIRNTKVKKGQKGNPKGRQNERVVIQYHYTQWPDMGVPEYALPVLTFVRRSSAARMPETGPVLVHCSAGVGRTGTYIVIDSMLQQIKDKSTVNVLGFLKHIRTQRNYLVQTEEQYIFIHDALLEAILGKETEVSSNQLHSYVNSILIPGVGGKTRLEKQFKLVTQCNAKYVECFSAQKECNKEKNRNSSVVPSERARVGLAPLPGMKGTDYINASYIMGYYRSNEFIITQHPLPHTTKDFWRMIWDHNAQIIVMLPDNQSLAEDEFVYWPSREESMNCEAFTVTLISKDRLCLSNEEQIIIHDFILEATQDDYVLEVRHFQCPKWPNPDAPISSTFELINVIKEEALTRDGPTIVHDEYGAVSAGMLCALTTLSQQLENENAVDVFQVAKMINLMRPGVFTDIEQYQFIYKAMLSLVSTKENGNGPMTVDKNGAVLIADESDPAESMESLV",
# PTPRZ_HUMAN
"MRILKRFLACIQLLCVCRLDWANGYYRQQRKLVEEIGWSYTGALNQKNWGKKYPTCNSPKQSPINIDEDLTQVNVNLKKLKFQGWDKTSLENTFIHNTGKTVEINLTNDYRVSGGVSEMVFKASKITFHWGKCNMSSDGSEHSLEGQKFPLEMQIYCFDADRFSSFEEAVKGKGKLRALSILFEVGTEENLDFKAIIDGVESVSRFGKQAALDPFILLNLLPNSTDKYYIYNGSLTSPPCTDTVDWIVFKDTVSISESQLAVFCEVLTMQQSGYVMLMDYLQNNFREQQYKFSRQVFSSYTGKEEIHEAVCSSEPENVQADPENYTSLLVTWERPRVVYDTMIEKFAVLYQQLDGEDQTKHEFLTDGYQDLGAILNNLLPNMSYVLQIVAICTNGLYGKYSDQLIVDMPTDNPELDLFPELIGTEEIIKEEEEGKDIEEGAIVNPGRDSATNQIRKKEPQISTTTHYNRIGTKYNEAKTNRSPTRGSEFSGKGDVPNTSLNSTSQPVTKLATEKDISLTSQTVTELPPHTVEGTSASLNDGSKTVLRSPHMNLSGTAESLNTVSITEYEEESLLTSFKLDTGAEDSSGSSPATSAIPFISENISQGYIFSSENPETITYDVLIPESARNASEDSTSSGSEESLKDPSMEGNVWFPSSTDITAQPDVGSGRESFLQTNYTEIRVDESEKTTKSFSAGPVMSQGPSVTDLEMPHYSTFAYFPTEVTPHAFTPSSRQQDLVSTVNVVYSQTTQPVYNGETPLQPSYSSEVFPLVTPLLLDNQILNTTPAASSSDSALHATPVFPSVDVSFESILSSYDGAPLLPFSSASFSSELFRHLHTVSQILPQVTSATESDKVPLHASLPVAGGDLLLEPSLAQYSDVLSTTHAASETLEFGSESGVLYKTLMFSQVEPPSSDAMMHARSSGPEPSYALSDNEGSQHIFTVSYSSAIPVHDSVGVTYQGSLFSGPSHIPIPKSSLITPTASLLQPTHALSGDGEWSGASSDSEFLLPDTDGLTALNISSPVSVAEFTYTTSVFGDDNKALSKSEIIYGNETELQIPSFNEMVYPSESTVMPNMYDNVNKLNASLQETSVSISSTKGMFPGSLAHTTTKVFDHEISQVPENNFSVQPTHTVSQASGDTSLKPVLSANSEPASSDPASSEMLSPSTQLLFYETSASFSTEVLLQPSFQASDVDTLLKTVLPAVPSDPILVETPKVDKISSTMLHLIVSNSASSENMLHSTSVPVFDVSPTSHMHSASLQGLTISYASEKYEPVLLKSESSHQVVPSLYSNDELFQTANLEINQAHPPKGRHVFATPVLSIDEPLNTLINKLIHSDEILTSTKSSVTGKVFAGIPTVASDTFVSTDHSVPIGNGHVAITAVSPHRDGSVTSTKLLFPSKATSELSHSAKSDAGLVGGGEDGDTDDDGDDDDDDRGSDGLSIHKCMSCSSYRESQEKVMNDSDTHENSLMDQNNPISYSLSENSEEDNRVTSVSSDSQTGMDRSPGKSPSANGLSQKHNDGKEENDIQTGSALLPLSPESKAWAVLTSDEESGSGQGTSDSLNENETSTDFSFADTNEKDADGILAAGDSEITPGFPQSPTSSVTSENSEVFHVSEAEASNSSHESRIGLAEGLESEKKAVIPLVIVSALTFICLVVLVGILIYWRKCFQTAHFYLEDSTSPRVISTPPTPIFPISDDVGAIPIKHFPKHVADLHASSGFTEEFETLKEFYQEVQSCTVDLGITADSSNHPDNKHKNRYINIVAYDHSRVKLAQLAEKDGKLTDYINANYVDGYNRPKAYIAAQGPLKSTAEDFWRMIWEHNVEVIVMITNLVEKGRRKCDQYWPADGSEEYGNFLVTQKSVQVLAYYTVRNFTLRNTKIKKGSQKGRPSGRVVTQYHYTQWPDMGVPEYSLPVLTFVRKAAYAKRHAVGPVVVHCSAGVGRTGTYIVLDSMLQQIQHEGTVNIFGFLKHIRSQRNYLVQTEEQYVFIHDTLVEAILSKETEVLDSHIHAYVNALLIPGPAGKTKLEKQFQLLSQSNIQQSDYSAALKQCNREKNRTSSIIPVERSRVGISSLSGEGTDYINASYIMGYYQSNEFIITQHPLLHTIKDFWRMIWDHNAQLVVMIPDGQNMAEDEFVYWPNKDEPINCESFKVTLMAEEHKCLSNEEKLIIQDFILEATQDDYVLEVRHFQCPKWPNPDSPISKTFELISVIKEEAANRDGPMIVHDEHGGVTAGTFCALTTLMHQLEKENSVDVYQVAKMINLMRPGVFADIEQYQFLYKVILSLVSTRQEENPSTSLDSNGAALPDGNIAESLESLV",
# PTPRK_HUMAN
"MDTTAAAALPAFVALLLLSPWPLLGSAQGQFSAGGCTFDDGPGACDYHQDLYDDFEWVHVSAQEPHYLPPEMPQGSYMIVDSSDHDPGEKARLQLPTMKENDTHCIDFSYLLYSQKGLNPGTLNILVRVNKGPLANPIWNVTGFTGRDWLRAELAVSTFWPNEYQVIFEAEVSGGRSGYIAIDDIQVLSYPCDKSPHFLRLGDVEVNAGQNATFQCIATGRDAVHNKLWLQRRNGEDIPVAQTKNINHRRFAASFRLQEVTKTDQDLYRCVTQSERGSGVSNFAQLIVREPPRPIAPPQLLGVGPTYLLIQLNANSIIGDGPIILKEVEYRMTSGSWTETHAVNAPTYKLWHLDPDTEYEIRVLLTRPGEGGTGLPGPPLITRTKCAEPMRTPKTLKIAEIQARRIAVDWESLGYNITRCHTFNVTICYHYFRGHNESKADCLDMDPKAPQHVVNHLPPYTNVSLKMILTNPEGRKESEETIIQTDEDVPGPVPVKSLQGTSFENKIFLNWKEPLDPNGIITQYEISYSSIRSFDPAVPVAGPPQTVSNLWNSTHHVFMHLHPGTTYQFFIRASTVKGFGPATAINVTTNISAPTLPDYEGVDASLNETATTITVLLRPAQAKGAPISAYQIVVEELHPHRTKREAGAMECYQVPVTYQNAMSGGAPYYFAAELPPGNLPEPAPFTVGDNRTYQGFWNPPLAPRKGYNIYFQAMSSVEKETKTQCVRIATKAATEEPEVIPDPAKQTDRVVKIAGISAGILVFILLLLVVILIVKKSKLAKKRKDAMGNTRQEMTHMVNAMDRSYADQSTLHAEDPLSITFMDQHNFSPRYENHSATAESSRLLDVPRYLCEGTESPYQTGQLHPAIRVADLLQHINLMKTSDSYGFKEEYESFFEGQSASWDVAKKDQNRAKNRYGNIIAYDHSRVILQPVEDDPSSDYINANYIDGYQRPSHYIATQGPVHETVYDFWRMIWQEQSACIVMVTNLVEVGRVKCYKYWPDDTEVYGDFKVTCVEMEPLAEYVVRTFTLERRGYNEIREVKQFHFTGWPDHGVPYHATGLLSFIRRVKLSNPPSAGPIVVHCSAGAGRTGCYIVIDIMLDMAEREGVVDIYNCVKALRSRRINMVQTEEQYIFIHDAILEACLCGETAIPVCEFKAAYFDMIRIDSQTNSSHLKDEFQTLNSVTPRLQAEDCSIACLPRNHDKNRFMDMLPPDRCLPFLITIDGESSNYINAALMDSYRQPAAFIVTQYPLPNTVKDFWRLVYDYGCTSIVMLNEVDLSQGCPQYWPEEGMLRYGPIQVECMSCSMDCDVINRIFRICNLTRPQEGYLMVQQFQYLGWASHREVPGSKRSFLKLILQVEKWQEECEEGEGRTIIHCLNGGGRSGMFCAIGIVVEMVKRQNVVDVFHAVKTLRNSKPNMVEAPEQYRFCYDVALEYLESS",
# PTPRM_HUMAN
"MRGLGTCLATLAGLLLTAAGETFSGGCLFDEPYSTCGYSQSEGDDFNWEQVNTLTKPTSDPWMPSGSFMLVNASGRPEGQRAHLLLPQLKENDTHCIDFHYFVSSKSNSPPGLLNVYVKVNNGPLGNPIWNISGDPTRTWNRAELAISTFWPNFYQVIFEVITSGHQGYLAIDEVKVLGHPCTRTPHFLRIQNVEVNAGQFATFQCSAIGRTVAGDRLWLQGIDVRDAPLKEIKVTSSRRFIASFNVVNTTKRDAGKYRCMIRTEGGVGISNYAELVVKEPPVPIAPPQLASVGATYLWIQLNANSINGDGPIVAREVEYCTASGSWNDRQPVDSTSYKIGHLDPDTEYEISVLLTRPGEGGTGSPGPALRTRTKCADPMRGPRKLEVVEVKSRQITIRWEPFGYNVTRCHSYNLTVHYCYQVGGQEQVREEVSWDTENSHPQHTITNLSPYTNVSVKLILMNPEGRKESQELIVQTDEDLPGAVPTESIQGSTFEEKIFLQWREPTQTYGVITLYEITYKAVSSFDPEIDLSNQSGRVSKLGNETHFLFFGLYPGTTYSFTIRASTAKGFGPPATNQFTTKISAPSMPAYELETPLNQTDNTVTVMLKPAHSRGAPVSVYQIVVEEERPRRTKKTTEILKCYPVPIHFQNASLLNSQYYFAAEFPADSLQAAQPFTIGDNKTYNGYWNTPLLPYKSYRIYFQAASRANGETKIDCVQVATKGAATPKPVPEPEKQTDHTVKIAGVIAGILLFVIIFLGVVLVMKKRKLAKKRKETMSSTRQEMTVMVNSMDKSYAEQGTNCDEAFSFMDTHNLNGRSVSSPSSFTMKTNTLSTSVPNSYYPDETHTMASDTSSLVQSHTYKKREPADVPYQTGQLHPAIRVADLLQHITQMKCAEGYGFKEEYESFFEGQSAPWDSAKKDENRMKNRYGNIIAYDHSRVRLQTIEGDTNSDYINGNYIDGYHRPNHYIATQGPMQETIYDFWRMVWHENTASIIMVTNLVEVGRVKCCKYWPDDTEIYKDIKVTLIETELLAEYVIRTFAVEKRGVHEIREIRQFHFTGWPDHGVPYHATGLLGFVRQVKSKSPPSAGPLVVHCSAGAGRTGCFIVIDIMLDMAEREGVVDIYNCVRELRSRRVNMVQTEEQYVFIHDAILEACLCGDTSVPASQVRSLYYDMNKLDPQTNSSQIKEEFRTLNMVTPTLRVEDCSIALLPRNHEKNRCMDILPPDRCLPFLITIDGESSNYINAALMDSYKQPSAFIVTQHPLPNTVKDFWRLVLDYHCTSVVMLNDVDPAQLCPQYWPENGVHRHGPIQVEFVSADLEEDIISRIFRIYNAARPQDGYRMVQQFQFLGWPMYRDTPVSKRSFLKLIRQVDKWQEEYNGGEGRTVVHCLNGGGRSGTFCAISIVCEMLRHQRTVDVFHAVKTLRNNKPNMVDLLDQYKFCYEVALEYLNSG",
# PTPRT_HUMAN
"MASLAALALSLLLRLQLPPLPGARAQSAAGGCSFDEHYSNCGYSVALGTNGFTWEQINTWEKPMLDQAVPTGSFMMVNSSGRASGQKAHLLLPTLKENDTHCIDFHYYFSSRDRSSPGALNVYVKVNGGPQGNPVWNVSGVVTEGWVKAELAISTFWPHFYQVIFESVSLKGHPGYIAVDEVRVLAHPCRKAPHFLRLQNVEVNVGQNATFQCIAGGKWSQHDKLWLQQWNGRDTALMVTRVVNHRRFSATVSVADTAQRSVSKYRCVIRSDGGSGVSNYAELIVKEPPTPIAPPELLAVGATYLWIKPNANSIIGDGPIILKEVEYRTTTGTWAETHIVDSPNYKLWHLDPDVEYEIRVLLTRPGEGGTGPPGPPLTTRTKCADPVHGPQNVEIVDIRARQLTLQWEPFGYAVTRCHSYNLTVQYQYVFNQQQYEAEEVIQTSSHYTLRGLRPFMTIRLRLLLSNPEGRMESEELVVQTEEDVPGAVPLESIQGGPFEEKIYIQWKPPNETNGVITLYEINYKAVGSLDPSADLSSQRGKVFKLRNETHHLFVGLYPGTTYSFTIKASTAKGFGPPVTTRIATKISAPSMPEYDTDTPLNETDTTITVMLKPAQSRGAPVSVYQLVVKEERLQKSRRAADIIECFSVPVSYRNASSLDSLHYFAAELKPANLPVTQPFTVGDNKTYNGYWNPPLSPLKSYSIYFQALSKANGETKINCVRLATKGASTQNSNTVEPEKQVDNTVKMAGVIAGLLMFIIILLGVMLTIKRRRNAYSYSYYLKLAKKQKETQSGAQREMGPVASADKPTTKLSASRNDEGFSSSSQDVNGFTDGSRGELSQPTLTIQTHPYRTCDPVEMSYPRDQFQPAIRVADLLQHITQMKRGQGYGFKEEYEALPEGQTASWDTAKEDENRNKNRYGNIISYDHSRVRLLVLDGDPHSDYINANYIDGYHRPRHYIATQGPMQETVKDFWRMIWQENSASIVMVTNLVEVGRVKCVRYWPDDTEVYGDIKVTLIETEPLAEYVIRTFTVQKKGYHEIRELRLFHFTSWPDHGVPCYATGLLGFVRQVKFLNPPEAGPIVVHCSAGAGRTGCFIAIDTMLDMAENEGVVDIFNCVRELRAQRVNLVQTEEQYVFVHDAILEACLCGNTAIPVCEFRSLYYNISRLDPQTNSSQIKDEFQTLNIVTPRVRPEDCSIGLLPRNHDKNRSMDVLPLDRCLPFLISVDGESSNYINAALMDSHKQPAAFVVTQHPLPNTVADFWRLVFDYNCSSVVMLNEMDTAQFCMQYWPEKTSGCYGPIQVEFVSADIDEDIIHRIFRICNMARPQDGYRIVQHLQYIGWPAYRDTPPSKRSLLKVVRRLEKWQEQYDGREGRTVVHCLNGGGRSGTFCAICSVCEMIQQQNIIDVFHIVKTLRNNKSNMVETLEQYKFVYEVALEYLSSF",
# PTPRN_HUMAN
"MRRPRRPGGLGGSGGLRLLLCLLLLSSRPGGCSAVSAHGCLFDRRLCSHLEVCIQDGLFGQCQVGVGQARPLLQVTSPVLQRLQGVLRQLMSQGLSWHDDLTQYVISQEMERIPRLRPPEPRPRDRSGLAPKRPGPAGELLLQDIPTGSAPAAQHRLPQPPVGKGGAGASSSLSPLQAELLPPLLEHLLLPPQPPHPSLSYEPALLQPYLFHQFGSRDGSRVSEGSPGMVSVGPLPKAEAPALFSRTASKGIFGDHPGHSYGDLPGPSPAQLFQDSGLLYLAQELPAPSRARVPRLPEQGSSSRAEDSPEGYEKEGLGDRGEKPASPAVQPDAALQRLAAVLAGYGVELRQLTPEQLSTLLTLLQLLPKGAGRNPGGVVNVGADIKKTMEGPVEGRDTAELPARTSPMPGHPTASPTSSEVQQVPSPVSSEPPKAARPPVTPVLLEKKSPLGQSQPTVAGQPSARPAAEEYGYIVTDQKPLSLAAGVKLLEILAEHVHMSSGSFINISVVGPALTFRIRHNEQNLSLADVTQQAGLVKSELEAQTGLQILQTGVGQREEAAAVLPQTAHSTSPMRSVLLTLVALAGVAGLLVALAVALCVRQHARQQDKERLAALGPEGAHGDTTFEYQDLCRQHMATKSLFNRAEGPPEPSRVSSVSSQFSDAAQASPSSHSSTPSWCEEPAQANMDISTGHMILAYMEDHLRNRDRLAKEWQALCAYQAEPNTCATAQGEGNIKKNRHPDFLPYDHARIKLKVESSPSRSDYINASPIIEHDPRMPAYIATQGPLSHTIADFWQMVWESGCTVIVMLTPLVEDGVKQCDRYWPDEGASLYHVYEVNLVSEHIWCEDFLVRSFYLKNVQTQETRTLTQFHFLSWPAEGTPASTRPLLDFRRKVNKCYRGRSCPIIVHCSDGAGRTGTYILIDMVLNRMAKGVKEIDIAATLEHVRDQRPGLVRSKDQFEFALTAVAEEVNAILKALPQ",
# PTPR2_HUMAN
"MGPPLPLLLLLLLLLPPRVLPAAPSSVPRGRQLPGRLGCLLEEGLCGASEACVNDGVFGRCQKVPAMDFYRYEVSPVALQRLRVALQKLSGTGFTWQDDYTQYVMDQELADLPKTYLRRPEASSPARPSKHSVGSERRYSREGGAALANALRRHLPFLEALSQAPASDVLARTHTAQDRPPAEGDDRFSESILTYVAHTSALTYPPGSRTQLREDLLPRTLGQLQPDELSPKVDSGVDRHHLMAALSAYAAQRPPAPPGEGSLEPQYLLRAPSRMPRPLLAPAAPQKWPSPLGDSEDPSSTGDGARIHTLLKDLQRQPAEVRGLSGLELDGMAELMAGLMQGVDHGVARGSPGRAALGESGEQADGPKATLRGDSFPDDGVQDDDDRLYQEVHRLSATLGGLLQDHGSRLLPGALPFARPLDMERKKSEHPESSLSSEEETAGVENVKSQTYSKDLLGQQPHSEPGAAAFGELQNQMPGPSKEEQSLPAGAQEALSDGLQLEVQPSEEEARGYIVTDRDPLRPEEGRRLVEDVARLLQVPSSAFADVEVLGPAVTFKVSANVQNVTTEDVEKATVDNKDKLEETSGLKILQTGVGSKSKLKFLPPQAEQEDSTKFIALTLVSLACILGVLLASGLIYCLRHSSQHRLKEKLSGLGGDPGADATAAYQELCRQRMATRPPDRPEGPHTSRISSVSSQFSDGPIPSPSARSSASSWSEEPVQSNMDISTGHMILSYMEDHLKNKNRLEKEWEALCAYQAEPNSSFVAQREENVPKNRSLAVLTYDHSRVLLKAENSHSHSDYINASPIMDHDPRNPAYIATQGPLPATVADFWQMVWESGCVVIVMLTPLAENGVRQCYHYWPDEGSNLYHIYEVNLVSEHIWCEDFLVRSFYLKNLQTNETRTVTQFHFLSWYDRGVPSSSRSLLDFRRKVNKCYRGRSCPIIVHCSDGAGRSGTYVLIDMVLNKMAKGAKEIDIAATLEHLRDQRPGMVQTKEQFEFALTAVAEEVNAILKALPQ",
# FIG4_HUMAN
"MPTAAAPIISSVQKLVLYETRARYFLVGSNNAETKYRVLKIDRTEPKDLVIIDDRHVYTQQEVRELLGRLDLGNRTKMGQKGSSGLFRAVSAFGVVGFVRFLEGYYIVLITKRRKMADIGGHAIYKVEDTNMIYIPNDSVRVTHPDEARYLRIFQNVDLSSNFYFSYSYDLSHSLQYNLTVLRMPLEMLKSEMTQNRQESFDIFEDEGLITQGGSGVFGICSEPYMKYVWNGELLDIIKSTVHRDWLLYIIHGFCGQSKLLIYGRPVYVTLIARRSSKFAGTRFLKRGANCEGDVANEVETEQILCDASVMSFTAGSYSSYVQVRGSVPLYWSQDISTMMPKPPITLDQADPFAHVAALHFDQMFQRFGSPIIILNLVKEREKRKHERILSEELVAAVTYLNQFLPPEHTIVYIPWDMAKYTKSKLCNVLDRLNVIAESVVKKTGFFVNRPDSYCSILRPDEKWNELGGCVIPTGRLQTGILRTNCVDCLDRTNTAQFMVGKCALAYQLYSLGLIDKPNLQFDTDAVRLFEELYEDHGDTLSLQYGGSQLVHRVKTYRKIAPWTQHSKDIMQTLSRYYSNAFSDADRQDSINLFLGVFHPTEGKPHLWELPTDFYLHHKNTMRLLPTRRSYTYWWTPEVIKHLPLPYDEVICAVNLKKLIVKKFHKYEEEIDIHNEFFRPYELSSFDDTFCLAMTSSARDFMPKTVGIDPSPFTVRKPDETGKSVLGNKSNREEAVLQRKTAASAPPPPSEEAVSSSSEDDSGTDREEEGSVSQRSTPVKMTDAGDSAKVTENVVQPMKELYGINLSDGLSEEDFSIYSRFVQLGQSQHKQDKNSQQPCSRCSDGVIKLTPISAFSQDNIYEVQPPRVDRKSTEIFQAHIQASQGIMQPLGKEDSSMYREYIRNRYL",
# SAC2_HUMAN
"MELFQAKDHYILQQGERALWCSRRDGGLQLRPATDLLLAWNPICLGLVEGVIGKIQLHSDLPWWLILIRQKALVGKLPGDHEVCKVTKIAVLSLSEMEPQDLELELCKKHHFGINKPEKIIPSPDDSKFLLKTFTHIKSNVSAPNKKKVKESKEKEKLERRLLEELLKMFMDSESFYYSLTYDLTNSVQRQSTGERDGRPLWQKVDDRFFWNKYMIQDLTEIGTPDVDFWIIPMIQGFVQIEELVVNYTESSDDEKSSPETPPQESTCVDDIHPRFLVALISRRSRHRAGMRYKRRGVDKNGNVANYVETEQLIHVHNHTLSFVQTRGSVPVFWSQVGYRYNPRPRLDRSEKETVAYFCAHFEEQLNIYKKQVIINLVDQAGREKIIGDAYLKQVLLFNNSHLTYVSFDFHEHCRGMKFENVQTLTDAIYDIILDMKWCWVDEAGVICKQEGIFRVNCMDCLDRTNVVQAAIARVVMEQQLKKLGVMPPEQPLPVKCNRIYQIMWANNGDSISRQYAGTAALKGDFTRTGERKLAGVMKDGVNSANRYYLNRFKDAYRQAVIDLMQGIPVTEDLYSIFTKEKEHEALHKENQRSHQELISQLLQSYMKLLLPDDEKFHGGWALIDCDPSLIDATHRDVDVLLLLSNSAYYVAYYDDEVDKVNQYQRLSLENLEKIEIGPEPTLFGKPKFSCMRLHYRYKEASGYFHTLRAVMRNPEEDGKDTLQCIAEMLQITKQAMGSDLPIIEKKLERKSSKPHEDIIGIRSQNQGSLAQGKNFLMSKFSSLNQKVKQTKSNVNIGNLRKLGNFTKPEMKVNFLKPNLKVNLWKSDSSLETMENTGVMDKVQAESDGDMSSDNDSYHSDEFLTNSKSDEDRQLANSLESVGPIDYVLPSCGIIASAPRLGSRSQSLSSTDSSVHAPSEITVAHGSGLGKGQESPLKKSPSAGDVHILTGFAKPMDIYCHRFVQDAQNKVTHLSETRSVSQQASQERNQMTNQVSNETQSESTEQTPSRPSQLDVSLSATGPQFLSVEPAHSVASQKTPTSASSMLELETGLHVTPSPSESSSSRAVSPFAKIRSSMVQVASITQAGLTHGINFAVSKVQKSPPEPEIINQVQQNELKKMFIQCQTRIIQI",
# SAC1_HUMAN
"MATAAYEQLKLHITPEKFYVEACDDGADDVLTIDRVSTEVTLAVKKDVPPSAVTRPIFGILGTIHLVAGNYLIVITKKIKVGEFFSHVVWKATDFDVLSYKKTMLHLTDIQLQDNKTFLAMLNHVLNVDGFYFSTTYDLTHTLQRLSNTSPEFQEMSLLERADQRFVWNGHLLRELSAQPEVHRFALPVLHGFITMHSCSINGKYFDWILISRRSCFRAGVRYYVRGIDSEGHAANFVETEQIVHYNGSKASFVQTRGSIPVFWSQRPNLKYKPLPQISKVANHMDGFQRHFDSQVIIYGKQVIINLINQKGSEKPLEQTFATMVSSLGSGMMRYIAFDFHKECKNMRWDRLSILLDQVAEMQDELSYFLVDSAGQVVANQEGVFRSNCMDCLDRTNVIQSLLARRSLQAQLQRLGVLHVGQKLEEQDEFEKIYKNAWADNANACAKQYAGTGALKTDFTRTGKRTHLGLIMDGWNSMIRYYKNNFSDGFRQDSIDLFLGNYSVDELESHSPLSVPRDWKFLALPIIMVVAFSMCIICLLMAGDTWTETLAYVLFWGVASIGTFFIILYNGKDFVDAPRLVQKEKID",
# SYNJ1_HUMAN
"MAFSKGFRIYHKLDPPPFSLIVETRHKEECLMFESGAVAVLSSAEKEAIKGTYSKVLDAYGLLGVLRLNLGDTMLHYLVLVTGCMSVGKIQESEVFRVTSTEFISLRIDSSDEDRISEVRKVLNSGNFYFAWSASGISLDLSLNAHRSMQEQTTDNRFFWNQSLHLHLKHYGVNCDDWLLRLMCGGVEIRTIYAAHKQAKACLISRLSCERAGTRFNVRGTNDDGHVANFVETEQVVYLDDSVSSFIQIRGSVPLFWEQPGLQVGSHRVRMSRGFEANAPAFDRHFRTLKNLYGKQIIVNLLGSKEGEHMLSKAFQSHLKASEHAADIQMVNFDYHQMVKGGKAEKLHSVLKPQVQKFLDYGFFYFNGSEVQRCQSGTVRTNCLDCLDRTNSVQAFLGLEMLAKQLEALGLAEKPQLVTRFQEVFRSMWSVNGDSISKIYAGTGALEGKAKLKDGARSVTRTIQNNFFDSSKQEAIDVLLLGNTLNSDLADKARALLTTGSLRVSEQTLQSASSKVLKSMCENFYKYSKPKKIRVCVGTWNVNGGKQFRSIAFKNQTLTDWLLDAPKLAGIQEFQDKRSKPTDIFAIGFEEMVELNAGNIVSASTTNQKLWAVELQKTISRDNKYVLLASEQLVGVCLFVFIRPQHAPFIRDVAVDTVKTGMGGATGNKGAVAIRMLFHTTSLCFVCSHFAAGQSQVKERNEDFIEIARKLSFPMGRMLFSHDYVFWCGDFNYRIDLPNEEVKELIRQQNWDSLIAGDQLINQKNAGQVFRGFLEGKVTFAPTYKYDLFSDDYDTSEKCRTPAWTDRVLWRRRKWPFDRSAEDLDLLNASFQDESKILYTWTPGTLLHYGRAELKTSDHRPVVALIDIDIFEVEAEERQNIYKEVIAVQGPPDGTVLVSIKSSLPENNFFDDALIDELLQQFASFGEVILIRFVEDKMWVTFLEGSSALNVLSLNGKELLNRTITIALKSPDWIKNLEEEMSLEKISIALPSSTSSTLLGEDAEVAADFDMEGDVDDYSAEVEELLPQHLQPSSSSGLGTSPSSSPRTSPCQSPTISEGPVPSLPIRPSRAPSRTPGPPSAQSSPIDAQPATPLPQKDPAQPLEPKRPPPPRPVAPPTRPAPPQRPPPPSGARSPAPTRKEFGGIGAPPSPGVARREMEAPKSPGTTRKDNIGRSQPSPQAGLAGPGPAGYSTARPTIPPRAGVISAPQSHARASAGRLTPESQSKTSETSKGSTFLPEPLKPQAAFPPQSSLPPPAQRLQEPLVPVAAPMPQSGPQPNLETPPQPPPRSRSSHSLPSEASSQPQVKTNGISDGKRESPLKIDPFEDLSFNLLAVSKAQLSVQTSPVPTPDPKRLIQLPSATQSNVLSSVSCMPTMPPIPARSQSQENMRSSPNPFITGLTRTNPFSDRTAAPGNPFRAKSEESEATSWFSKEEPVTISPFPSLQPLGHNKSRASSSLDGFKDSFDLQGQSTLKISNPKGWVTFEEEEDFGVKGKSKSACSDLLGNQPSSFSGSNLTLNDDWNKGTNVSFCVLPSRRPPPPPVPLLPPGTSPPVDPFTTLASKASPTLDFTER",
# SYNJ2_HUMAN
"MALSKGLRLLGRLGAEGDCSVLLEARGRDDCLLFEAGTVATLAPEEKEVIKGQYGKLTDAYGCLGELRLKSGGTSLSFLVLVTGCTSVGRIPDAEIYKITATDFYPLQEEAKEEERLIALKKILSSGVFYFSWPNDGSRFDLTVRTQKQGDDSSEWGNSFFWNQLLHVPLRQHQVSCCDWLLKIICGVVTIRTVYASHKQAKACLVSRVSCERTGTRFHTRGVNDDGHVSNFVETEQMIYMDDGVSSFVQIRGSVPLFWEQPGLQVGSHHLRLHRGLEANAPAFDRHMVLLKEQYGQQVVVNLLGSRGGEEVLNRAFKKLLWASCHAGDTPMINFDFHQFAKGGKLEKLETLLRPQLKLHWEDFDVFTKGENVSPRFQKGTLRMNCLDCLDRTNTVQSFIALEVLHLQLKTLGLSSKPIVDRFVESFKAMWSLNGHSLSKVFTGSRALEGKAKVGKLKDGARSMSRTIQSNFFDGVKQEAIKLLLVGDVYGEEVADKGGMLLDSTALLVTPRILKAMTERQSEFTNFKRIRIAMGTWNVNGGKQFRSNVLRTAELTDWLLDSPQLSGATDSQDDSSPADIFAVGFEEMVELSAGNIVNASTTNKKMWGEQLQKAISRSHRYILLTSAQLVGVCLYIFVRPYHVPFIRDVAIDTVKTGMGGKAGNKGAVGIRFQFHSTSFCFICSHLTAGQSQVKERNEDYKEITQKLCFPMGRNVFSHDYVFWCGDFNYRIDLTYEEVFYFVKRQDWKKLLEFDQLQLQKSSGKIFKDFHEGAINFGPTYKYDVGSAAYDTSDKCRTPAWTDRVLWWRKKHPFDKTAGELNLLDSDLDVDTKVRHTWSPGALQYYGRAELQASDHRPVLAIVEVEVQEVDVGARERVFQEVSSFQGPLDATVVVNLQSPTLEEKNEFPEDLRTELMQTLGSYGTIVLVRINQGQMLVTFADSHSALSVLDVDGMKVKGRAVKIRPKTKDWLKGLREEIIRKRDSMAPVSPTANSCLLEENFDFTSLDYESEGDILEDDEDYLVDEFNQPGVSDSELGGDDLSDVPGPTALAPPSKSPALTKKKQHPTYKDDADLVELKRELEAVGEFRHRSPSRSLSVPNRPRPPQPPQRPPPPTGLMVKKSASDASISSGTHGQYSILQTARLLPGAPQQPPKARTGISKPYNVKQIKTTNAQEAEAAIRCLLEARGGASEEALSAVAPRDLEASSEPEPTPGAAKPETPQAPPLLPRRPPPRVPAIKKPTLRRTGKPLSPEEQFEQQTVHFTIGPPETSVEAPPVVTAPRVPPVPKPRTFQPGKAAERPSHRKPASDEAPPGAGASVPPPLEAPPLVPKVPPRRKKSAPAAFHLQVLQSNSQLLQGLTYNSSDSPSGHPPAAGTVFPQGDFLSTSSATSPDSDGTKAMKPEAAPLLGDYQDPFWNLLHHPKLLNNTWLSKSSDPLDSGTRSPKRDPIDPVSAGASAAKAELPPDHEHKTLGHWVTISDQEKRTALQVFDPLAKT",
# SSU72_HUMAN
"MPSSPLRVAVVCSSNQNRSMEAHNILSKRGFSVRSFGTGTHVKLPGPAPDKPNVYDFKTTYDQMYNDLLRKDKELYTQNGILHMLDRNKRIKPRPERFQNCKDLFDLILTCEERVYDQVVEDLNSREQETCQPVHVVNVDIQDNHEEATLGAFLICELCQCIQHTEDMENEIDELLQEFEEKSGRTFLHTVCFY",
# MPIP1_HUMAN
"MELGPEPPHRRRLLFACSPPPASQPVVKALFGASAAGGLSPVTNLTVTMDQLQGLGSDYEQPLEVKNNSNLQRMGSSESTDSGFCLDSPGPLDSKENLENPMRRIHSLPQKLLGCSPALKRSHSDSLDHDIFQLIDPDENKENEAFEFKKPVRPVSRGCLHSHGLQEGKDLFTQRQNSAPARMLSSNERDSSEPGNFIPLFTPQSPVTATLSDEDDGFVDLLDGENLKNEEETPSCMASLWTAPLVMRTTNLDNRCKLFDSPSLCSSSTRSVLKRPERSQEESPPGSTKRRKSMSGASPKESTNPEKAHETLHQSLSLASSPKGTIENILDNDPRDLIGDFSKGYLFHTVAGKHQDLKYISPEIMASVLNGKFANLIKEFVIIDCRYPYEYEGGHIKGAVNLHMEEEVEDFLLKKPIVPTDGKRVIVVFHCEFSSERGPRMCRYVRERDRLGNEYPKLHYPELYVLKGGYKEFFMKCQSYCEPPSYRPMHHEDFKEDLKKFRTKSRTWAGEKSKREMYSRLKKL",
# MPIP2_HUMAN
"MEVPQPEPAPGSALSPAGVCGGAQRPGHLPGLLLGSHGLLGSPVRAAASSPVTTLTQTMHDLAGLGSETPKSQVGTLLFRSRSRLTHLSLSRRASESSLSSESSESSDAGLCMDSPSPMDPHMAEQTFEQAIQAASRIIRNEQFAIRRFQSMPVRLLGHSPVLRNITNSQAPDGRRKSEAGSGAASSSGEDKENDGFVFKMPWKPTHPSSTHALAEWASRREAFAQRPSSAPDLMCLSPDRKMEVEELSPLALGRFSLTPAEGDTEEDDGFVDILESDLKDDDAVPPGMESLISAPLVKTLEKEEEKDLVMYSKCQRLFRSPSMPCSVIRPILKRLERPQDRDTPVQNKRRRSVTPPEEQQEAEEPKARVLRSKSLCHDEIENLLDSDHRELIGDYSKAFLLQTVDGKHQDLKYISPETMVALLTGKFSNIVDKFVIVDCRYPYEYEGGHIKTAVNLPLERDAESFLLKSPIAPCSLDKRVILIFHCEFSSERGPRMCRFIRERDRAVNDYPSLYYPEMYILKGGYKEFFPQHPNFCEPQDYRPMNHEAFKDELKTFRLKTRSWAGERSRRELCSRLQDQ",
# MPIP3_HUMAN
"MSTELFSSTREEGSSGSGPSFRSNQRKMLNLLLERDTSFTVCPDVPRTPVGKFLGDSANLSILSGGTPKRCLDLSNLSSGEITATQLTTSADLDETGHLDSSGLQEVHLAGMNHDQHLMKCSPAQLLCSTPNGLDRGHRKRDAMCSSSANKENDNGNLVDSEMKYLGSPITTVPKLDKNPNLGEDQAEEISDELMEFSLKDQEAKVSRSGLYRSPSMPENLNRPRLKQVEKFKDNTIPDKVKKKYFSGQGKLRKGLCLKKTVSLCDITITQMLEEDSNQGHLIGDFSKVCALPTVSGKHQDLKYVNPETVAALLSGKFQGLIEKFYVIDCRYPYEYLGGHIQGALNLYSQEELFNFFLKKPIVPLDTQKRIIIVFHCEFSSERGPRMCRCLREEDRSLNQYPALYYPELYILKGGYRDFFPEYMELCEPQSYCPMHHQDHKTELLRCRSQSKVQEGERQLREQIALLVKDMSP",
# EYA1_HUMAN
"MEMQDLTSPHSRLSGSSESPSGPKLGNSHINSNSMTPNGTEVKTEPMSSSETASTTADGSLNNFSGSAIGSSSFSPRPTHQFSPPQIYPSNRPYPHILPTPSSQTMAAYGQTQFTTGMQQATAYATYPQPGQPYGISSYGALWAGIKTEGGLSQSQSPGQTGFLSYGTSFSTPQPGQAPYSYQMQGSSFTTSSGIYTGNNSLTNSSGFNSSQQDYPSYPSFGQGQYAQYYNSSPYPAHYMTSSNTSPTTPSTNATYQLQEPPSGITSQAVTDPTAEYSTIHSPSTPIKDSDSDRLRRGSDGKSRGRGRRNNNPSPPPDSDLERVFIWDLDETIIVFHSLLTGSYANRYGRDPPTSVSLGLRMEEMIFNLADTHLFFNDLEECDQVHIDDVSSDDNGQDLSTYNFGTDGFPAAATSANLCLATGVRGGVDWMRKLAFRYRRVKEIYNTYKNNVGGLLGPAKREAWLQLRAEIEALTDSWLTLALKALSLIHSRTNCVNILVTTTQLIPALAKVLLYGLGIVFPIENIYSATKIGKESCFERIIQRFGRKVVYVVIGDGVEEEQGAKKHAMPFWRISSHSDLMALHHALELEYL",
# EYA2_HUMAN
"MVELVISPSLTVNSDCLDKLKFNRADAAVWTLSDRQGITKSAPLRVSQLFSRSCPRVLPRQPSTAMAAYGQTQYSAGIQQATPYTAYPPPAQAYGIPSYSIKTEDSLNHSPGQSGFLSYGSSFSTSPTGQSPYTYQMHGTTGFYQGGNGLGNAAGFGSVHQDYPSYPGFPQSQYPQYYGSSYNPPYVPASSICPSPLSTSTYVLQEASHNVPNQSSESLAGEYNTHNGPSTPAKEGDTDRPHRASDGKLRGRSKRSSDPSPAGDNEIERVFVWDLDETIIIFHSLLTGTFASRYGKDTTTSVRIGLMMEEMIFNLADTHLFFNDLEDCDQIHVDDVSSDDNGQDLSTYNFSADGFHSSAPGANLCLGSGVHGGVDWMRKLAFRYRRVKEMYNTYKNNVGGLIGTPKRETWLQLRAELEALTDLWLTHSLKALNLINSRPNCVNVLVTTTQLIPALAKVLLYGLGSVFPIENIYSATKTGKESCFERIMQRFGRKAVYVVIGDGVEEEQGAKKHNMPFWRISCHADLEALRHALELEYL",
# EYA3_HUMAN
"MEEEQDLPEQPVKKAKMQESGEQTISQVSNPDVSDQKPETSSLASNLPMSEEIMTCTDYIPRSSNDYTSQMYSAKPYAHILSVPVSETAYPGQTQYQTLQQTQPYAVYPQATQTYGLPPFGALWPGMKPESGLIQTPSPSQHSVLTCTTGLTTSQPSPAHYSYPIQASSTNASLISTSSTIANIPAAAVASISNQDYPTYTILGQNQYQACYPSSSFGVTGQTNSDAESTTLAATTYQSEKPSVMAPAPAAQRLSSGDPSTSPSLSQTTPSKDTDDQSRKNMTSKNRGKRKADATSSQDSELERVFLWDLDETIIIFHSLLTGSYAQKYGKDPTVVIGSGLTMEEMIFEVADTHLFFNDLEECDQVHVEDVASDDNGQDLSNYSFSTDGFSGSGGSGSHGSSVGVQGGVDWMRKLAFRYRKVREIYDKHKSNVGGLLSPQRKEALQRLRAEIEVLTDSWLGTALKSLLLIQSRKNCVNVLITTTQLVPALAKVLLYGLGEIFPIENIYSATKIGKESCFERIVSRFGKKVTYVVIGDGRDEEIAAKQHNMPFWRITNHGDLVSLHQALELDFL",
# EYA4_HUMAN
"MEDSQDLNEQSVKKTCTESDVSQSQNSRSMEMQDLASPHTLVGGGDTPGSSKLEKSNLSSTSVTTNGTGGENMTVLNTADWLLSCNTPSSATMSLLAVKTEPLNSSETTATTGDGALDTFTGSVITSSGYSPRSAHQYSPQLYPSKPYPHILSTPAAQTMSAYAGQTQYSGMQQPAVYTAYSQTGQPYSLPTYDLGVMLPAIKTESGLSQTQSPLQSGCLSYSPGFSTPQPGQTPYSYQMPGSSFAPSSTIYANNSVSNSTNFSGSQQDYPSYTAFGQNQYAQYYSASTYGAYMTSNNTADGTPSSTSTYQLQESLPGLTNQPGEFDTMQSPSTPIKDLDERTCRSSGSKSRGRGRKNNPSPPPDSDLERVFVWDLDETIIVFHSLLTGSYAQKYGKDPPMAVTLGLRMEEMIFNLADTHLFFNDLEECDQVHIDDVSSDDNGQDLSTYSFATDGFHAAASSANLCLPTGVRGGVDWMRKLAFRYRRVKELYNTYKNNVGGLLGPAKRDAWLQLRAEIEGLTDSWLTNALKSLSIISTRSNCINVLVTTTQLIPALAKVLLYSLGGAFPIENIYSATKIGKESCFERIMQRFGRKVVYVVIGDGVEEEQAAKKHNMPFWRISSHSDLLALHQALELEYL",
# CTSL2_HUMAN
"MRLRTRKASQQSNQIQTQRTARAKRKYSEVDDSLPSGGEKPSKNETGLLSSIKKFIKGSTPKEERENPSKRSRIERDIDNNLITSTPRAGEKPNKQISRVRRKSQVNGEAGSYEMTNQHVKQNGKLEDNPSSGSPPRTTLLGTIFSPVFNFFSPANKNGTSGSDSPGQAVEAEEIVKQLDMEQVDEITTSTTTSTNGAAYSNQAVQVRPSLNNGLEEAEETVNRDIPPLTAPVTPDSGYSSAHAEATYEEDWEVFDPYYFIKHVPPLTEEQLNRKPALPLKTRSTPEFSLVLDLDETLVHCSLNELEDAALTFPVLFQDVIYQVYVRLRPFFREFLERMSQMYEIILFTASKKVYADKLLNILDPKKQLVRHRLFREHCVCVQGNYIKDLNILGRDLSKTIIIDNSPQAFAYQLSNGIPIESWFMDKNDNELLKLIPFLEKLVELNEDVRPHIRDRFRLHDLLPPD",
# CNEP1_HUMAN
"MMRTQCLLGLRTFVAFAAKLWSFFIYLLRRQIRTVIQYQTVRYDILPLSPVSRNRLAQVKRKILVLDLDETLIHSHHDGVLRPTVRPGTPPDFILKVVIDKHPVRFFVHKRPHVDFFLEVVSQWYELVVFTASMEIYGSAVADKLDNSRSILKRRYYRQHCTLELGSYIKDLSVVHSDLSSIVILDNSPGAYRSHPDNAIPIKSWFSDPSDTALLNLLPMLDALRFTADVRSVLSRNLHQHRLW",
# CTDP1_HUMAN
"MEVPAAGRVPAEGAPTAAVAEVRCPGPAPLRLLEWRVAAGAAVRIGSVLAVFEAAASAQSSGASQSRVASGGCVRPARPERRLRSERAGVVRELCAQPGQVVAPGAVLVRLEGCSHPVVMKGLCAECGQDLTQLQSKNGKQQVPLSTATVSMVHSVPELMVSSEQAEQLGREDQQRLHRNRKLVLMVDLDQTLIHTTEQHCQQMSNKGIFHFQLGRGEPMLHTRLRPHCKDFLEKIAKLYELHVFTFGSRLYAHTIAGFLDPEKKLFSHRILSRDECIDPFSKTGNLRNLFPCGDSMVCIIDDREDVWKFAPNLITVKKYVYFQGTGDMNAPPGSRESQTRKKVNHSRGTEVSEPSPPVRDPEGVTQAPGVEPSNGLEKPARELNGSEAATPRDSPRPGKPDERDIWPPAQAPTSSQELAGAPEPQGSCAQGGRVAPGQRPAQGATGTDLDFDLSSDSESSSESEGTKSSSSASDGESEGKRGRQKPKAAPEGAGALAQGSSLEPGRPAAPSLPGEAEPGAHAPDKEPELGGQEEGERDGLCGLGNGCADRKEAETESQNSELSGVTAGESLDQSMEEEEEEDTDEDDHLIYLEEILVRVHTDYYAKYDRYLNKEIEEAPDIRKIVPELKSKVLADVAIIFSGLHPTNFPIEKTREHYHATALGAKILTRLVLSPDAPDRATHLIAARAGTEKVLQAQECGHLHVVNPDWLWSCLERWDKVEEQLFPLRDDHTKAQRENSPAAFPDREGVPPTALFHPMPVLPKAQPGPEVRIYDSNTGKLIRTGARGPPAPSSSLPIRQEPSSFRAVPPPQPQMFGEELPDAQDGEQPGPSRRKRQPSMSETMPLYTLCKEDLESMDKEVDDILGEGSDDSDSEKRRPEEQEEEPQPRKPGTRRERTLGAPASSERSAAGGRGPRGHKRKLNEEDAASESSRESSNEDEGSSSEADEMAKALEAELNDLM",
# SYCP1_HUMAN
"MEKQKPFALFVPPRSSSSQVSAVKPQTLGGDSTFFKSFNKCTEDDFEFPFAKTNLSKNGENIDSDPALQKVNFLPVLEQVGNSDCHYQEGLKDSDLENSEGLSRVYSKLYKEAEKIKKWKVSTEAELRQKESKLQENRKIIEAQRKAIQELQFGNEKVSLKLEEGIQENKDLIKENNATRHLCNLLKETCARSAEKTKKYEYEREETRQVYMDLNNNIEKMITAFEELRVQAENSRLEMHFKLKEDYEKIQHLEQEYKKEINDKEKQVSLLLIQITEKENKMKDLTFLLEESRDKVNQLEEKTKLQSENLKQSIEKQHHLTKELEDIKVSLQRSVSTQKALEEDLQIATKTICQLTEEKETQMEESNKARAAHSFVVTEFETTVCSLEELLRTEQQRLEKNEDQLKILTMELQKKSSELEEMTKLTNNKEVELEELKKVLGEKETLLYENKQFEKIAEELKGTEQELIGLLQAREKEVHDLEIQLTAITTSEQYYSKEVKDLKTELENEKLKNTELTSHCNKLSLENKELTQETSDMTLELKNQQEDINNNKKQEERMLKQIENLQETETQLRNELEYVREELKQKRDEVKCKLDKSEENCNNLRKQVENKNKYIEELQQENKALKKKGTAESKQLNVYEIKVNKLELELESAKQKFGEITDTYQKEIEDKKISEENLLEEVEKAKVIADEAVKLQKEIDKRCQHKIAEMVALMEKHKHQYDKIIEERDSELGLYKSKEQEQSSLRASLEIELSNLKAELLSVKKQLEIEREEKEKLKREAKENTATLKEKKDKKTQTFLLETPEIYWKLDSKAVPSQTVSRNFTSVDHGISKDKRDYLWTSAKNTLSTPLPKAYTVKTPTKPKLQQRENLNIPIEESKKKRKMAFEFDINSDSSETTDLLSMVSEEETLKTLYRNNNPPASHLCVKTPKKAPSSLTTPGSTLKFGAIRKMREDRWAVIAKMDRKKKLKEAEKLFV",
# CTDS1_HUMAN
"MDSSAVITQISKEEARGPLRGKGDQKSAASQKPRSRGILHSLFCCVCRDDGEALPAHSGAPLLVEENGAIPKQTPVQYLLPEAKAQDSDKICVVIDLDETLVHSSFKPVNNADFIIPVEIDGVVHQVYVLKRPHVDEFLQRMGELFECVLFTASLAKYADPVADLLDKWGAFRARLFRESCVFHRGNYVKDLSRLGRDLRRVLILDNSPASYVFHPDNAVPVASWFDNMSDTELHDLLPFFEQLSRVDDVYSVLRQPRPGS",
# RISC_HUMAN
"MELALRRSPVPRWLLLLPLLLGLNAGAVIDWPTEEGKEVWDYVTVRKDAYMFWWLYYATNSCKNFSELPLVMWLQGGPGGSSTGFGNFEEIGPLDSDLKPRKTTWLQAASLLFVDNPVGTGFSYVNGSGAYAKDLAMVASDMMVLLKTFFSCHKEFQTVPFYIFSESYGGKMAAGIGLELYKAIQRGTIKCNFAGVALGDSWISPVDSVLSWGPYLYSMSLLEDKGLAEVSKVAEQVLNAVNKGLYREATELWGKAEMIIEQNTDGVNFYNILTKSTPTSTMESSLEFTQSHLVCLCQRHVRHLQRDALSQLMNGPIRKKLKIIPEDQSWGGQATNVFVNMEEDFMKPVISIVDELLEAGINVTVYNGQLDLIVDTMGQEAWVRKLKWPELPKFSQLKWKALYSDPKSLETSAFVKSYKNLAFYWILKAGHMVPSDQGDMALKMMRLVTQQE",
# CTDS2_HUMAN
"MEHGSIITQARREDALVLTKQGLVSKSSPKKPRGRNIFKALFCCFRAQHVGQSSSSTELAAYKEEANTIAKSDLLQCLQYQFYQIPGTCLLPEVTEEDQGRICVVIDLDETLVHSSFKPINNADFIVPIEIEGTTHQVYVLKRPYVDEFLRRMGELFECVLFTASLAKYADPVTDLLDRCGVFRARLFRESCVFHQGCYVKDLSRLGRDLRKTLILDNSPASYIFHPENAVPVQSWFDDMADTELLNLIPIFEELSGAEDVYTSLGQLRAP",
# SCP2_HUMAN
"MSSSPWEPATLRRVFVVGVGMTKFVKPGAENSRDYPDLAEEAGKKALADAQIPYSAVDQACVGYVFGDSTCGQRAIYHSLGMTGIPIINVNNNCATGSTALFMARQLIQGGVAECVLALGFEKMSKGSLGIKFSDRTIPTDKHVDLLINKYGLSAHPVAPQMFGYAGKEHMEKYGTKIEHFAKIGWKNHKHSVNNPYSQFQDEYSLDEVMASKEVFDFLTILQCCPTSDGAAAAILASEAFVQKYGLQSKAVEILAQEMMTDLPSSFEEKSIIKMVGFDMSKEAARKCYEKSGLTPNDIDVIELHDCFSTNELLTYEALGLCPEGQGATLVDRGDNTYGGKWVINPSGGLISKGHPLGATGLAQCAELCWQLRGEAGKRQVPGAKVALQHNLGIGGAVVVTLYKMGFPEAASSFRTHQIEAVPTSSASDGFKANLVFKEIEKKLEEEGEQFVKKIGGIFAFKVKDGPGGKEATWVVDVKNGKGSVLPNSDKKADCTITMADSDFLALMTGKMNPQSAFFQGKLKITGNMGLAMKLQNLQLQPGNAKL",
# SYCP2_HUMAN
"MPIRPDLQQLEKCIDDALRKNDFKPLKTLLQIDICEDVKIKCSKQFFHKVDNLICRELNKEDIHNVSAILVSVGRCGKNISVLGQAGLLTMIKQGLIQKMVAWFEKSKDIIQSQGNSKDEAVLNMIEDLVDLLLVIHDVSDEGKKQVVESFVPRICSLVIDSRVNICIQQEIIKKMNAMLDKMPQDARKILSNQEMLILMSSMGERILDAGDYDLQVGIVEALCRMTTEKQRQELAHQWFSMDFIAKAFKRIKDSEFETDCRIFLNLVNGMLGDKRRVFTFPCLSAFLDKYELQIPSDEKLEEFWIDFNLGSQTLSFYIAGDNDDHQWEAVTVPEEKVQIYSIEVRESKKLLTIILKNTVKISKREGKELLLYFDASLEITNVTQKIFGATKHRESIRKQGISVAKTSLHILFDASGSQILVPESQISPVGEELVSLKEKSKSPKEFAKPSKYIKNSDKGNRNNSQLEKTTPSKRKMSEASMIVSGADRYTMRSPVLFSNTSIPPRRRRIKPPLQMTSSAEKPSVSQTSENRVDNAASLKSRSSEGRHRRDNIDKHIKTAKCVENTENKNVEFPNQNFSELQDVIPDSQAAEKRDHTILPGVLDNICGNKIHSKWACWTPVTNIELCNNQRASTSSGDTLNQDIVINKKLTKQKSSSSISDHNSEGTGKVKYKKEQTDHIKIDKAEVEVCKKHNQQQNHPKYSGQKNTENAKQSDWPVESETTFKSVLLNKTIEESLIYRKKYILSKDVNTATCDKNPSASKNVQSHRKAEKELTSELNSWDSKQKKMREKSKGKEFTNVAESLISQINKRYKTKDDIKSTRKLKESLINSGFSNKPVVQLSKEKVQKKSYRKLKTTFVNVTSECPVNDVYNFNLNGADDPIIKLGIQEFQATAKEACADRSIRLVGPRNHDELKSSVKTKDKKIITNHQKKNLFSDTETEYRCDDSKTDISWLREPKSKPQLIDYSRNKNVKNHKSGKSRSSLEKGQPSSKMTPSKNITKKMDKTIPEGRIRLPRKATKTKKNYKDLSNSESECEQEFSHSFKENIPVKEENIHSRMKTVKLPKKQQKVFCAETEKELSKQWKNSSLLKDAIRDNCLDLSPRSLSGSPSSIEVTRCIEKITEKDFTQDYDCITKSISPYPKTSSLESLNSNSGVGGTIKSPKNNEKNFLCASESCSPIPRPLFLPRHTPTKSNTIVNRKKISSLVLTQETQNSNSYSDVSSYSSEERFMEIESPHINENYIQSKREESHLASSLSKSSEGREKTWFDMPCDATHVSGPTQHLSRKRIYIEDNLSNSNEVEMEEKGERRANLLPKKLCKIEDADHHIHKMSESVSSLSTNDFSIPWETWQNEFAGIEMTYETYERLNSEFKRRNNIRHKMLSYFTTQSWKTAQQHLRTMNHQSQDSRIKKLDKFQFIIIEELENFEKDSQSLKDLEKEFVDFWEKIFQKFSAYQKSEQQRLHLLKTSLAKSVFCNTDSEETVFTSEMCLMKEDMKVLQDRLLKDMLEEELLNVRRELMSVFMSHERNANV",
# CTDSL_HUMAN
"MDGPAIITQVTNPKEDEGRLPGAGEKASQCNVSLKKQRSRSILSSFFCCFRDYNVEAPPPSSPSVLPPLVEENGGLQKGDQRQVIPIPSPPAKYLLPEVTVLDYGKKCVVIDLDETLVHSSFKPISNADFIVPVEIDGTIHQVYVLKRPHVDEFLQRMGQLFECVLFTASLAKYADPVADLLDRWGVFRARLFRESCVFHRGNYVKDLSRLGRELSKVIIVDNSPASYIFHPENAVPVQSWFDDMTDTELLDLIPFFEGLSREDDVYSMLHRLCNR",
# SYCP3_HUMAN
"MVSSGKKYSRKSGKPSVEDQFTRAYDFETEDKKDLSGSEEDVIEGKTAVIEKRRKKRSSAGVVEDMGGEVQNMLEGVGVDINKALLAKRKRLEMYTKASLKTSNQKIEHVWKTQQDQRQKLNQEYSQQFLTLFQQWDLDMQKAEEQEEKILNMFRQQQKILQQSRIVQSQRLKTIKQLYEQFIKSMEELEKNHDNLLTGAQNEFKKEMAMLQKKIMMETQQQEIASVRKSLQSMLF",
# TIM50_HUMAN
"MAASAAVFSRLRSGLRLGSRGLCTRLATPPRRAPDQAAEIGSRGSTKAQGPQQQPGSEGPSYAKKVALWLAGLLGAGGTVSVVYIFGNNPVDENGAKIPDEFDNDPILVQQLRRTYKYFKDYRQMIIEPTSPCLLPDPLQEPYYQPPYTLVLELTGVLLHPEWSLATGWRFKKRPGIETLFQQLAPLYEIVIFTSETGMTAFPLIDSVDPHGFISYRLFRDATRYMDGHHVKDISCLNRDPARVVVVDCKKEAFRLQPYNGVALRPWDGNSDDRVLLDLSAFLKTIALNGVEDVRTVLEHYALEDDPLAAFKQRQSRLEQEEQQRLAELSKSNKQNLFLGSLTSRLWPRSKQP",
# UBCP1_HUMAN
"MALPIIVKWGGQEYSVTTLSEDDTVLDLKQFLKTLTGVLPERQKLLGLKVKGKPAENDVKLGALKLKPNTKIMMMGTREESLEDVLGPPPDNDDVVNDFDIEDEVVEVENREENLLKISRRVKEYKVEILNPPREGKKLLVLDVDYTLFDHRSCAETGVELMRPYLHEFLTSAYEDYDIVIWSATNMKWIEAKMKELGVSTNANYKITFMLDSAAMITVHTPRRGLIDVKPLGVIWGKFSEFYSKKNTIMFDDIGRNFLMNPQNGLKIRPFMKAHLNRDKDKELLKLTQYLKEIAKLDDFLDLNHKYWERYLSKKQGQ",
# HDHD5_HUMAN
"MAAWGCVAALGAARGLCWRAARAAAGLQGRPARRCYAVGPAQSPPTFGFLLDIDGVLVRGHRVIPAALKAFRRLVNSQGQLRVPVVFVTNAGNILQHSKAQELSALLGCEVDADQVILSHSPMKLFSEYHEKRMLVSGQGPVMENAQGLGFRNVVTVDELRMAFPLLDMVDLERRLKTTPLPRNDFPRIEGVLLLGEPVRWETSLQLIMDVLLSNGSPGAGLATPPYPHLPVLASNMDLLWMAEAKMPRFGHGTFLLCLETIYQKVTGKELRYEGLMGKPSILTYQYAEDLIRRQAERRGWAAPIRKLYAVGDNPMSDVYGANLFHQYLQKATHDGAPELGAGGTRQQQPSASQSCISILVCTGVYNPRNPQSTEPVLGGGEPPFHGHRDLCFSPGLMEASHVVNDVNEAVQLVFRKEGWALE",
# HDHD2_HUMAN
"MAACRALKAVLVDLSGTLHIEDAAVPGAQEALKRLRGASVIIRFVTNTTKESKQDLLERLRKLEFDISEDEIFTSLTAARSLLERKQVRPMLLVDDRALPDFKGIQTSDPNAVVMGLAPEHFHYQILNQAFRLLLDGAPLIAIHKARYYKRKDGLALGPGPFVTALEYATDTKATVVGKPEKTFFLEALRGTGCEPEEAVMIGDDCRDDVGGAQDVGMLGILVKTGKYRASDEEKINPPPYLTCESFPHAVDHILQHLL",
# LHPP_HUMAN
"MAPWGKRLAGVRGVLLDISGVLYDSGAGGGTAIAGSVEAVARLKRSRLKVRFCTNESQKSRAELVGQLQRLGFDISEQEVTAPAPAACQILKEQGLRPYLLIHDGVRSEFDQIDTSNPNCVVIADAGESFSYQNMNNAFQVLMELEKPVLISLGKGRYYKETSGLMLDVGPYMKALEYACGIKAEVVGKPSPEFFKSALQAIGVEAHQAVMIGDDIVGDVGGAQRCGMRALQVRTGKFRPSDEHHPEVKADGYVDNLAEAVDLLLQHADK",
# PLPP_HUMAN
"MARCERLRGAALRDVLGRAQGVLFDCDGVLWNGERAVPGAPELLERLARAGKAALFVSNNSRRARPELALRFARLGFGGLRAEQLFSSALCAARLLRQRLPGPPDAPGAVFVLGGEGLRAELRAAGLRLAGDPSAGDGAAPRVRAVLVGYDEHFSFAKLREACAHLRDPECLLVATDRDPWHPLSDGSRTPGTGSLAAAVETASGRQALVVGKPSPYMFECITENFSIDPARTLMVGDRLETDILFGHRCGMTTVLTLTGVSRLEEAQAYLAAGQHDLVPHYYVESIADLTEGLED",
# PGP_HUMAN
"MAAAEAGGDDARCVRLSAERAQALLADVDTLLFDCDGVLWRGETAVPGAPEALRALRARGKRLGFITNNSSKTRAAYAEKLRRLGFGGPAGPGASLEVFGTAYCTALYLRQRLAGAPAPKAYVLGSPALAAELEAVGVASVGVGPEPLQGEGPGDWLHAPLEPDVRAVVVGFDPHFSYMKLTKALRYLQQPGCLLVGTNMDNRLPLENGRFIAGTGCLVRAVEMAAQRQADIIGKPSRFIFDCVSQEYGINPERTVMVGDRLDTDILLGATCGLKTILTLTGVSTLGDVKNNQESDCVSKKKMVPDFYVDSIADLLPALQG",
# F261_HUMAN
"MSPEMGELTQTRLQKIWIPHSSGSSRLQRRRGSSIPQFTNSPTMVIMVGLPARGKTYISTKLTRYLNWIGTPTKVFNLGQYRREAVSYKNYEFFLPDNMEALQIRKQCALAALKDVHNYLSHEEGHVAVFDATNTTRERRSLILQFAKEHGYKVFFIESICNDPGIIAENIRQVKLGSPDYIDCDREKVLEDFLKRIECYEVNYQPLDEELDSHLSYIKIFDVGTRYMVNRVQDHIQSRTVYYLMNIHVTPRSIYLCRHGESELNIRGRIGGDSGLSVRGKQYAYALANFIQSQGISSLKVWTSHMKRTIQTAEALGVPYEQWKALNEIDAGVCEEMTYEEIQEHYPEEFALRDQDKYRYRYPKGESYEDLVQRLEPVIMELERQENVLVICHQAVMRCLLAYFLDKSSDELPYLKCPLHTVLKLTPVAYGCKVESIYLNVEAVNTHREKPENVDITREPEEALDTVPAHY",
# F262_HUMAN
"MSGASSSEQNNNSYETKTPNLRMSEKKCSWASYMTNSPTLIVMIGLPARGKTYVSKKLTRYLNWIGVPTKVFNLGVYRREAVKSYKSYDFFRHDNEEAMKIRKQCALVALEDVKAYLTEENGQIAVFDATNTTRERRDMILNFAEQNSFKVFFVESVCDDPDVIAANILEVKVSSPDYPERNRENVMEDFLKRIECYKVTYRPLDPDNYDKDLSFIKVINVGQRFLVNRVQDYIQSKIVYYLMNIHVQPRTIYLCRHGESEFNLLGKIGGDSGLSVRGKQFAQALRKFLEEQEITDLKVWTSQLKRTIQTAESLGVPYEQWKILNEIDAGVCEEMTYAEIEKRYPEEFALRDQEKYLYRYPGGESYQDLVQRLEPVIMELERQGNVLVISHQAVMRCLLAYFLDKGADELPYLRCPLHTIFKLTPVAYGCKVETIKLNVEAVNTHRDKPTNNFPKNQTPVRMRRNSFTPLSSSNTIRRPRNYSVGSRPLKPLSPLRAQDMQEGAD",
# F263_HUMAN
"MPLELTQSRVQKIWVPVDHRPSLPRSCGPKLTNSPTVIVMVGLPARGKTYISKKLTRYLNWIGVPTKVFNVGEYRREAVKQYSSYNFFRPDNEEAMKVRKQCALAALRDVKSYLAKEGGQIAVFDATNTTRERRHMILHFAKENDFKAFFIESVCDDPTVVASNIMEVKISSPDYKDCNSAEAMDDFMKRISCYEASYQPLDPDKCDRDLSLIKVIDVGRRFLVNRVQDHIQSRIVYYLMNIHVQPRTIYLCRHGENEHNLQGRIGGDSGLSSRGKKFASALSKFVEEQNLKDLRVWTSQLKSTIQTAEALRLPYEQWKALNEIDAGVCEELTYEEIRDTYPEEYALREQDKYYYRYPTGESYQDLVQRLEPVIMELERQENVLVICHQAVLRCLLAYFLDKSAEEMPYLKCPLHTVLKLTPVAYGCRVESIYLNVESVCTHRERSEDAKKGPNPLMRRNSVTPLASPEPTKKPRINSFEEHVASTSAALPSCLPPEVPTQLPGQNMKGSRSSADSSRKH",
# F264_HUMAN
"MASPRELTQNPLKKIWMPYSNGRPALHACQRGVCMTNCPTLIVMVGLPARGKTYISKKLTRYLNWIGVPTREFNVGQYRRDVVKTYKSFEFFLPDNEEGLKIRKQCALAALRDVRRFLSEEGGHVAVFDATNTTRERRATIFNFGEQNGYKTFFVESICVDPEVIAANIVQVKLGSPDYVNRDSDEATEDFMRRIECYENSYESLDEDLDRDLSYIKIMDVGQSYVVNRVADHIQSRIVYYLMNIHVTPRSIYLCRHGESELNLKGRIGGDPGLSPRGREFAKSLAQFISDQNIKDLKVWTSQMKRTIQTAEALGVPYEQWKVLNEIDAGVCEEMTYEEIQDNYPLEFALRDQDKYRYRYPKGESYEDLVQRLEPVIMELERQENVLVICHQAVMRCLLAYFLDKAAEQLPYLKCPLHTVLKLTPVAYGCKVESIFLNVAAVNTHRDRPQNVDISRPPEEALVTVPAHQ",
# PMGE_HUMAN
"MSKYKLIMLRHGEGAWNKENRFCSWVDQKLNSEGMEEARNCGKQLKALNFEFDLVFTSVLNRSIHTAWLILEELGQEWVPVESSWRLNERHYGALIGLNREQMALNHGEEQVRLWRRSYNVTPPPIEESHPYYQEIYNDRRYKVCDVPLDQLPRSESLKDVLERLLPYWNERIAPEVLRGKTILISAHGNSSRALLKHLEGISDEDIINITLPTGVPILLELDENLRAVGPHQFLGDQEAIQAAIKKVEDQGKVKQAKK",
# PGAM1_HUMAN
"MAAYKLVLIRHGESAWNLENRFSGWYDADLSPAGHEEAKRGGQALRDAGYEFDICFTSVQKRAIRTLWTVLDAIDQMWLPVVRTWRLNERHYGGLTGLNKAETAAKHGEAQVKIWRRSYDVPPPPMEPDHPFYSNISKDRRYADLTEDQLPSCESLKDTIARALPFWNEEIVPQIKEGKRVLIAAHGNSLRGIVKHLEGLSEEAIMELNLPTGIPIVYELDKNLKPIKPMQFLGDEETVRKAMEAVAAQGKAKK",
# PGAM2_HUMAN
"MATHRLVMVRHGESTWNQENRFCGWFDAELSEKGTEEAKRGAKAIKDAKMEFDICYTSVLKRAIRTLWAILDGTDQMWLPVVRTWRLNERHYGGLTGLNKAETAAKHGEEQVKIWRRSFDIPPPPMDEKHPYYNSISKERRYAGLKPGELPTCESLKDTIARALPFWNEEIVPQIKAGKRVLIAAHGNSLRGIVKHLEGMSDQAIMELNLPTGIPIVYELNKELKPTKPMQFLGDEETVRKAMEAVAAQGKAK",
# PGAM4_HUMAN
"MAAYKLVLIRHGESTWNLENRFSCWYDADLSPAGHEEAKRGGQALRDAGYEFDICLTSVQKRVIRTLWTVLDAIDQMWLPVVRTWRLNERHYGGLTGLNKAETAAKHGEAQVKIWRRSYDVPPPPMEPDHPFYSNISKDRRYADLTEDQLPSYESPKDTIARALPFWNEEIVPQIKEGKRVLIAAHGNSLQGIAKHVEGLSEEAIMELNLPTGIPIVYELDKNLKPIKPMQFLGDEETVCKAIEAVAAQGKAKK",
# PGAM5_HUMAN
"MAFRQALQLAACGLAGGSAAVLFSAVAVGKPRAGGDAEPRPAEPPAWAGGARPGPGVWDPNWDRREPLSLINVRKRNVESGEEELASKLDHYKAKATRHIFLIRHSQYHVDGSLEKDRTLTPLGREQAELTGLRLASLGLKFNKIVHSSMTRAIETTDIISRHLPGVCKVSTDLLREGAPIEPDPPVSHWKPEAVQYYEDGARIEAAFRNYIHRADARQEEDSYEIFICHANVIRYIVCRALQFPPEGWLRLSLNNGSITHLVIRPNGRVALRTLGDTGFMPPDKITRS",
# UBS3A_HUMAN
"MAAGETQLYAKVSNKLKSRSSPSLLEPLLAMGFPVHTALKALAATGRKTAEEALAWLHDHCNDPSLDDPIPQEYALFLCPTGPLLEKLQEFWRESKRQCAKNRAHEVFPHVTLCDFFTCEDQKVECLYEALKRAGDRLLGSFPTAVPLALHSSISYLGFFVSGSPADVIREFAMTFATEASLLAGTSVSRFWIFSQVPGHGPNLRLSNLTRASFVSHYILQKYCSVKPCTKQLHLTLAHKFYPHHQRTLEQLARAIPLGHSCQWTAALYSRDMRFVHYQTLRALFQYKPQNVDELTLSPGDYIFVDPTQQDEASEGWVIGISQRTGCRGFLPENYTDRASESDTWVKHRMYTFSLATDLNSRKDGEASSRCSGEFLPQTARSLSSLQALQATVARKSVLVVRHGERVDQIFGKAWLQQCSTPDGKYYRPDLNFPCSLPRRSRGIKDFENDPPLSSCGIFQSRIAGDALLDSGIRISSVFASPALRCVQTAKLILEELKLEKKIKIRVEPGIFEWTKWEAGKTTPTLMSLEELKEANFNIDTDYRPAFPLSALMPAESYQEYMDRCTASMVQIVNTCPQDTGVILIVSHGSTLDSCTRPLLGLPPRECGDFAQLVRKIPSLGMCFCEENKEEGKWELVNPPVKTLTHGANAAFNWRNWISGN",
# UBS3B_HUMAN
"MAQYGHPSPLGMAAREELYSKVTPRRNRQQRPGTIKHGSALDVLLSMGFPRARAQKALASTGGRSVQAACDWLFSHVGDPFLDDPLPREYVLYLRPTGPLAQKLSDFWQQSKQICGKNKAHNIFPHITLCQFFMCEDSKVDALGEALQTTVSRWKCKFSAPLPLELYTSSNFIGLFVKEDSAEVLKKFAADFAAEAASKTEVHVEPHKKQLHVTLAYHFQASHLPTLEKLAQNIDVKLGCDWVATIFSRDIRFANHETLQVIYPYTPQNDDELELVPGDFIFMSPMEQTSTSEGWIYGTSLTTGCSGLLPENYITKADECSTWIFHGSYSILNTSSSNSLTFGDGVLERRPYEDQGLGETTPLTIICQPMQPLRVNSQPGPQKRCLFVCRHGERMDVVFGKYWLSQCFDAKGRYIRTNLNMPHSLPQRSGGFRDYEKDAPITVFGCMQARLVGEALLESNTIIDHVYCSPSLRCVQTAHNILKGLQQENHLKIRVEPGLFEWTKWVAGSTLPAWIPPSELAAANLSVDTTYRPHIPISKLVVSESYDTYISRSFQVTKEIISECKSKGNNILIVAHASSLEACTCQLQGLSPQNSKDFVQMVRKIPYLGFCSCEELGETGIWQLTDPPILPLTHGPTGGFNWRETLLQE",
# TIGAR_HUMAN
"MARFALTVVRHGETRFNKEKIIQGQGVDEPLSETGFKQAAAAGIFLNNVKFTHAFSSDLMRTKQTMHGILERSKFCKDMTVKYDSRLRERKYGVVEGKALSELRAMAKAAREECPVFTPPGGETLDQVKMRGIDFFEFLCQLILKEADQKEQFSQGSPSNCLETSLAEIFPLGKNHSSKVNSDSGIPGLAASVLVVSHGAYMRSLFDYFLTDLKCSLPATLSRSELMSVTPNTGMSLFIINFEEGREVKPTVQCICMNLQDHLNGLTETR",
# PPAL_HUMAN
"MAGKRSGWSRAALLQLLLGVNLVVMPPTRARSLRFVTLLYRHGDRSPVKTYPKDPYQEEEWPQGFGQLTKEGMLQHWELGQALRQRYHGFLNTSYHRQEVYVRSTDFDRTLMSAEANLAGLFPPNGMQRFNPNISWQPIPVHTVPITEDRLLKFPLGPCPRYEQLQNETRQTPEYQNESSRNAQFLDMVANETGLTDLTLETVWNVYDTLFCEQTHGLRLPPWASPQTMQRLSRLKDFSFRFLFGIYQQAEKARLQGGVLLAQIRKNLTLMATTSQLPKLLVYSAHDTTLVALQMALDVYNGEQAPYASCHIFELYQEDSGNFSVEMYFRNESDKAPWPLSLPGCPHRCPLQDFLRLTEPVVPKDWQQECQLASGPADTEVIVALAVCGSILFLLIVLLLTVLFRMQAQPPGYRHVADGEDHA",
# PPAP_HUMAN
"MRAAPLLLARAASLSLGFLFLLFFWLDRSVLAKELKFVTLVFRHGDRSPIDTFPTDPIKESSWPQGFGQLTQLGMEQHYELGEYIRKRYRKFLNESYKHEQVYIRSTDVDRTLMSAMTNLAALFPPEGVSIWNPILLWQPIPVHTVPLSEDQLLYLPFRNCPRFQELESETLKSEEFQKRLHPYKDFIATLGKLSGLHGQDLFGIWSKVYDPLYCESVHNFTLPSWATEDTMTKLRELSELSLLSLYGIHKQKEKSRLQGGVLVNEILNHMKRATQIPSYKKLIMYSAHDTTVSGLQMALDVYNGLLPPYASCHLTELYFEKGEYFVEMYYRNETQHEPYPLMLPGCSPSCPLERFAELVGPVIPQDWSTECMTTNSHQGTEDSTD",
# PPAT_HUMAN
"MAGLGFWGHPAGPLLLLLLLVLPPRALPEGPLVFVALVFRHGDRAPLASYPMDPHKEVASTLWPRGLGQLTTEGVRQQLELGRFLRSRYEAFLSPEYRREEVYIRSTDFDRTLESAQANLAGLFPEAAPGSPEARWRPIPVHTVPVAEDKLLRFPMRSCPRYHELLREATEAAEYQEALEGWTGFLSRLENFTGLSLVGEPLRRAWKVLDTLMCQQAHGLPLPAWASPDVLRTLAQISALDIGAHVGPPRAAEKAQLTGGILLNAILANFSRVQRLGLPLKMVMYSAHDSTLLALQGALGLYDGHTPPYAACLGFEFRKHLGNPAKDGGNVTVSLFYRNDSAHLPLPLSLPGCPAPCPLGRFYQLTAPARPPAHGVSCHGPYEAAIPPAPVVPLLAGAVAVLVALSLGLGLLAWRPGCLRALGGPV",
# PPA6_HUMAN
"MITGVFSMRLWTPVGVLTSLAYCLHQRRVALAELQEADGQCPVDRSLLKLKMVQVVFRHGARSPLKPLPLEEQVEWNPQLLEVPPQTQFDYTVTNLAGGPKPYSPYDSQYHETTLKGGMFAGQLTKVGMQQMFALGERLRKNYVEDIPFLSPTFNPQEVFIRSTNIFRNLESTRCLLAGLFQCQKEGPIIIHTDEADSEVLYPNYQSCWSLRQRTRGRRQTASLQPGISEDLKKVKDRMGIDSSDKVDFFILLDNVAAEQAHNLPSCPMLKRFARMIEQRAVDTSLYILPKEDRESLQMAVGPFLHILESNLLKAMDSATAPDKIRKLYLYAAHDVTFIPLLMTLGIFDHKWPPFAVDLTMELYQHLESKEWFVQLYYHGKEQVPRGCPDGLCPLDMFLNAMSVYTLSPEKYHALCSQTQVMEVGNEE",
# MINP1_HUMAN
"MLRAPGCLLRTSVAPAAALAAALLSSLARCSLLEPRDPVASSLSPYFGTKTRYEDVNPVLLSGPEAPWRDPELLEGTCTPVQLVALIRHGTRYPTVKQIRKLRQLHGLLQARGSRDGGASSTGSRDLGAALADWPLWYADWMDGQLVEKGRQDMRQLALRLASLFPALFSRENYGRLRLITSSKHRCMDSSAAFLQGLWQHYHPGLPPPDVADMEFGPPTVNDKLMRFFDHCEKFLTEVEKNATALYHVEAFKTGPEMQNILKKVAATLQVPVNDLNADLIQVAFFTCSFDLAIKGVKSPWCDVFDIDDAKVLEYLNDLKQYWKRGYGYTINSRSSCTLFQDIFQHLDKAVEQKQRSQPISSPVILQFGHAETLLPLLSLMGYFKDKEPLTAYNYKKQMHRKFRSGLIVPYASNLIFVLYHCENAKTPKEQFRVQMLLNEKVLPLAYSQETVSFYEDLKNHYKDILQSCQTSEECELARANSTSDEL",
# VIP1_HUMAN
"MWSLTASEGESTTAHFFLGAGDEGLGTRGIGMRPEESDSELLEDEEDEVPPEPQIIVGICAMTKKSKSKPMTQILERLCRFDYLTVVILGEDVILNEPVENWPSCHCLISFHSKGFPLDKAVAYSKLRNPFLINDLAMQYYIQDRREVYRILQEEGIDLPRYAVLNRDPARPEECNLIEGEDQVEVNGAVFPKPFVEKPVSAEDHNVYIYYPSSAGGGSQRLFRKIGSRSSVYSPESSVRKTGSYIYEEFMPTDGTDVKVYTVGPDYAHAEARKSPALDGKVERDSEGKEIRYPVMLTAMEKLVARKVCVAFKQTVCGFDLLRANGHSFVCDVNGFSFVKNSMKYYDDCAKILGNTIMRELAPQFQIPWSIPTEAEDIPIVPTTSGTMMELRCVIAIIRHGDRTPKQKMKMEVKHPRFFALFEKHGGYKTGKLKLKRPEQLQEVLDITRLLLAELEKEPGGEIEEKTGKLEQLKSVLEMYGHFSGINRKVQLTYYPHGVKASNEGQDPQRETLAPSLLLVLKWGGELTPAGRVQAEELGRAFRCMYPGGQGDYAGFPGCGLLRLHSTFRHDLKIYASDEGRVQMTAAAFAKGLLALEGELTPILVQMVKSANMNGLLDSDGDSLSSCQHRVKARLHHILQQDAPFGPEDYDQLAPTRSTSLLNSMTIIQNPVKVCDQVFALIENLTHQIRERMQDPRSVDLQLYHSETLELMLQRWSKLERDFRQKSGRYDISKIPDIYDCVKYDVQHNGSLGLQGTAELLRLSKALADVVIPQEYGISREEKLEIAVGFCLPLLRKILLDLQRTHEDESVNKLHPLCYLRYSRGVLSPGRHVRTRLYFTSESHVHSLLSVFRYGGLLDETQDAQWQRALDYLSAISELNYMTQIVIMLYEDNTQDPLSEERFHVELHFSPGVKGVEEEGSAPAGCGFRPASSENEEMKTNQGSMENLCPGKASDEPDRALQTSPQPPEGPGLPRRSPLIRNRKAGSMEVLSETSSSRPGGYRLFSSSRPPTEMKQSGLGSQCTGLFSTTVLGGSSSAPNLQDYARSHGKKLPPASLKHRDELLFVPAVKRFSVSFAKHPTNGFEGCSMVPTIYPLETLHNALSLRQVSEFLSRVCQRHTDAQAQASAALFDSMHSSQASDNPFSPPRTLHSPPLQLQQRSEKPPWYSSGPSSTVSSAGPSSPTTVDGNSQFGFSDQPSLNSHVAEEHQGLGLLQETPGSGAQELSIEGEQELFEPNQSPQVPPMETSQPYEEVSQPCQEVPDISQPCQDISEALSQPCQKVPDISQQCQENHDNGNHTCQEVPHISQPCQKSSQLCQKVSEEVCQLCLENSEEVSQPCQGVSVEVGKLVHKFHVGVGSLVQETLVEVGSPAEEIPEEVIQPYQEFSVEVGRLAQETSAINLLSQGIPEIDKPSQEFPEEIDLQAQEVPEEIN",
# VIP2_HUMAN
"MSEAPRFFVGPEDTEINPGNYRHFFHHADEDDEEEDDSPPERQIVVGICSMAKKSKSKPMKEILERISLFKYITVVVFEEEVILNEPVENWPLCDCLISFHSKGFPLDKAVAYAKLRNPFVINDLNMQYLIQDRREVYSILQAEGILLPRYAILNRDPNNPKECNLIEGEDHVEVNGEVFQKPFVEKPVSAEDHNVYIYYPTSAGGGSQRLFRKIGSRSSVYSPESNVRKTGSYIYEEFMPTDGTDVKVYTVGPDYAHAEARKSPALDGKVERDSEGKEVRYPVILNAREKLIAWKVCLAFKQTVCGFDLLRANGQSYVCDVNGFSFVKNSMKYYDDCAKILGNIVMRELAPQFHIPWSIPLEAEDIPIVPTTSGTMMELRCVIAVIRHGDRTPKQKMKMEVRHQKFFDLFEKCDGYKSGKLKLKKPKQLQEVLDIARQLLMELGQNNDSEIEENKPKLEQLKTVLEMYGHFSGINRKVQLTYLPHGCPKTSSEEEDSRREEPSLLLVLKWGGELTPAGRVQAEELGRAFRCMYPGGQGDYAGFPGCGLLRLHSTYRHDLKIYASDEGRVQMTAAAFAKGLLALEGELTPILVQMVKSANMNGLLDSDSDSLSSCQQRVKARLHEILQKDRDFTAEDYEKLTPSGSISLIKSMHLIKNPVKTCDKVYSLIQSLTSQIRHRMEDPKSSDIQLYHSETLELMLRRWSKLEKDFKTKNGRYDISKIPDIYDCIKYDVQHNGSLKLENTMELYRLSKALADIVIPQEYGITKAEKLEIAKGYCTPLVRKIRSDLQRTQDDDTVNKLHPVYSRGVLSPERHVRTRLYFTSESHVHSLLSILRYGALCNESKDEQWKRAMDYLNVVNELNYMTQIVIMLYEDPNKDLSSEERFHVELHFSPGAKGCEEDKNLPSGYGYRPASRENEGRRPFKIDNDDEPHTSKRDEVDRAVILFKPMVSEPIHIHRKSPLPRSRKTATNDEESPLSVSSPEGTGTWLHYTSGVGTGRRRRRSGEQITSSPVSPKSLAFTSSIFGSWQQVVSENANYLRTPRTLVEQKQNPTVGSHCAGLFSTSVLGGSSSAPNLQDYARTHRKKLTSSGCIDDATRGSAVKRFSISFARHPTNGFELYSMVPSICPLETLHNALSLKQVDEFLASIASPSSDVPRKTAEISSTALRSSPIMRKKVSLNTYTPAKILPTPPATLKSTKASSKPATSGPSSAVVPNTSSRKKNITSKTETHEHKKNTGKKK",
# PXYP1_HUMAN
"MLFRNRFLLLLALAALLAFVSLSLQFFHLIPVSTPKNGMSSKSRKRIMPDPVTEPPVTDPVYEALLYCNIPSVAERSMEGHAPHHFKLVSVHVFIRHGDRYPLYVIPKTKRPEIDCTLVANRKPYHPKLEAFISHMSKGSGASFESPLNSLPLYPNHPLCEMGELTQTGVVQHLQNGQLLRDIYLKKHKLLPNDWSADQLYLETTGKSRTLQSGLALLYGFLPDFDWKKIYFRHQPSALFCSGSCYCPVRNQYLEKEQRRQYLLRLKNSQLEKTYGEMAKIVDVPTKQLRAANPIDSMLCHFCHNVSFPCTRNGCVDMEHFKVIKTHQIEDERERREKKLYFGYSLLGAHPILNQTIGRMQRATEGRKEELFALYSAHDVTLSPVLSALGLSEARFPRFAARLIFELWQDREKPSEHSVRILYNGVDVTFHTSFCQDHHKRSPKPMCPLENLVRFVKRDMFVALGGSGTNYYDACHREGF",
# ILKAP_HUMAN
"MDLFGDLPEPERSPRPAAGKEAQKGPLLFDDLPPASSTDSGSGGPLLFDDLPPASSGDSGSLATSISQMVKTEGKGAKRKTSEEEKNGSEELVEKKVCKASSVIFGLKGYVAERKGEREEMQDAHVILNDITEECRPPSSLITRVSYFAVFDGHGGIRASKFAAQNLHQNLIRKFPKGDVISVEKTVKRCLLDTFKHTDEEFLKQASSQKPAWKDGSTATCVLAVDNILYIANLGDSRAILCRYNEESQKHAALSLSKEHNPTQYEERMRIQKAGGNVRDGRVLGVLEVSRSIGDGQYKRCGVTSVPDIRRCQLTPNDRFILLACDGLFKVFTPEEAVNFILSCLEDEKIQTREGKSAADARYEAACNRLANKAVQRGSADNVTVMVVRIGH",
# PLPP6_HUMAN
"MPSPRRSMEGRPLGVSASSSSSSPGSPAHGGGGGGSRFEFQSLLSSRATAVDPTCARLRASESPVHRRGSFPLAAAGPSQSPAPPLPEEDRMDLNPSFLGIALRSLLAIDLWLSKKLGVCAGESSSWGSVRPLMKLLEISGHGIPWLLGTLYCLCRSDSWAGREVLMNLLFALLLDLLLVALIKGLVRRRRPAHNQMDMFVTLSVDKYSFPSGHATRAALMSRFILNHLVLAIPLRVLVVLWAFVLGLSRVMLGRHNVTDVAFGFFLGYMQYSIVDYCWLSPHNAPVLFLLWSQR",
# PDP1_HUMAN
"MPAPTQLFFPLIRNCELSRIYGTACYCHHKHLCCSSSYIPQSRLRYTPHPAYATFCRPKENWWQYTQGRRYASTPQKFYLTPPQVNSILKANEYSFKVPEFDGKNVSSILGFDSNQLPANAPIEDRRSAATCLQTRGMLLGVFDGHAGCACSQAVSERLFYYIAVSLLPHETLLEIENAVESGRALLPILQWHKHPNDYFSKEASKLYFNSLRTYWQELIDLNTGESTDIDVKEALINAFKRLDNDISLEAQVGDPNSFLNYLVLRVAFSGATACVAHVDGVDLHVANTGDSRAMLGVQEEDGSWSAVTLSNDHNAQNERELERLKLEHPKSEAKSVVKQDRLLGLLMPFRAFGDVKFKWSIDLQKRVIESGPDQLNDNEYTKFIPPNYHTPPYLTAEPEVTYHRLRPQDKFLVLATDGLWETMHRQDVVRIVGEYLTGMHHQQPIAVGGYKVTLGQMHGLLTERRTKMSSVFEDQNAATHLIRHAVGNNEFGTVDHERLSKMLSLPEELARMYRDDITIIVVQFNSHVVGAYQNQE",
# PDP2_HUMAN
"MSSTVSYWILNSTRNSIATLQGGRRLYSRYVSNRNKLKWRLFSRVPPTLNSSPCGGFTLCKAYRHTSTEEDDFHLQLSPEQINEVLRAGETTHKILDLESRVPNSVLRFESNQLAANSPVEDRRGVASCLQTNGLMFGIFDGHGGHACAQAVSERLFYYVAVSLMSHQTLEHMEGAMESMKPLLPILHWLKHPGDSIYKDVTSVHLDHLRVYWQELLDLHMEMGLSIEEALMYSFQRLDSDISLEIQAPLEDEVTRNLSLQVAFSGATACMAHVDGIHLHVANAGDCRAILGVQEDNGMWSCLPLTRDHNAWNQAELSRLKREHPESEDRTIIMEDRLLGVLIPCRAFGDVQLKWSKELQRSILERGFNTEALNIYQFTPPHYYTPPYLTAEPEVTYHRLRPQDKFLVLASDGLWDMLSNEDVVRLVVGHLAEADWHKTDLAQRPANLGLMQSLLLQRKASGLHEADQNAATRLIRHAIGNNEYGEMEAERLAAMLTLPEDLARMYRDDITVTVVYFNSESIGAYYKGG",
# PHLP1_HUMAN
"MEPAAAATVQRLPELGREDRASAPAAAAAAAAAAAAAAAALAAAAGGGRSPEPALTPAAPSGGNGSGSGAREEAPGEAPPGPLPGRAGGAGRRRRRGAPQPIAGGAAPVPGAGGGANSLLLRRGRLKRNLSAAAAAASSSSSSSAAAASHSPGAAGLPASCSASASLCTRSLDRKTLLLKHRQTLQLQPSDRDWVRHQLQRGCVHVFDRHMASTYLRPVLCTLDTTAGEVAARLLQLGHKGGGVVKVLGQGPGAAAAREPAEPPPEAGPRLAPPEPRDSEVPPARSAPGAFGGPPRAPPADLPLPVGGPGGWSRRASPAPSDSSPGEPFVGGPVSSPRAPRPVVSDTESFSLSPSAESVSDRLDPYSSGGGSSSSSEELEADAASAPTGVPGQPRRPGHPAQPLPLPQTASSPQPQQKAPRAIDSPGGAVREGSCEEKAAAAVAPGGLQSTPGRSGVTAEKAPPPPPPPTLYVQLHGETTRRLEAEEKPLQIQNDYLFQLGFGELWRVQEEGMDSEIGCLIRFYAGKPHSTGSSERIQLSGMYNVRKGKMQLPVNRWTRRQVILCGTCLIVSSVKDSLTGKMHVLPLIGGKVEEVKKHQHCLAFSSSGPQSQTYYICFDTFTEYLRWLRQVSKVASQRISSVDLSCCSLEHLPANLFYSQDLTHLNLKQNFLRQNPSLPAARGLNELQRFTKLKSLNLSNNHLGDFPLAVCSIPTLAELNVSCNALRSVPAAVGVMHNLQTFLLDGNFLQSLPAELENMKQLSYLGLSFNEFTDIPEVLEKLTAVDKLCMSGNCVETLRLQALRKMPHIKHVDLRLNVIRKLIADEVDFLQHVTQLDLRDNKLGDLDAMIFNNIEVLHCERNQLVTLDICGYFLKALYASSNELVQLDVYPVPNYLSYMDVSRNRLENVPEWVCESRKLEVLDIGHNQICELPARLFCNSSLRKLLAGHNQLARLPERLERTSVEVLDVQHNQLLELPPNLLMKADSLRFLNASANKLESLPPATLSEETNSILQELYLTNNSLTDKCVPLLTGHPHLKILHMAYNRLQSFPASKMAKLEELEEIDLSGNKLKAIPTTIMNCRRMHTVIAHSNCIEVFPEVMQLPEIKCVDLSCNELSEVTLPENLPPKLQELDLTGNPRLVLDHKTLELLNNIRCFKIDQPSTGDASGAPAVWSHGYTEASGVKNKLCVAALSVNNFCDNREALYGVFDGDRNVEVPYLLQCTMSDILAEELQKTKNEEEYMVNTFIVMQRKLGTAGQKLGGAAVLCHIKHDPVDPGGSFTLTSANVGKCQTVLCRNGKPLPLSRSYIMSCEEELKRIKQHKAIITEDGKVNGVTESTRILGYTFLHPSVVPRPHVQSVLLTPQDEFFILGSKGLWDSLSVEEAVEAVRNVPDALAAAKKLCTLAQSYGCHDSISAVVVQLSVTEDSFCCCELSAGGAVPPPSPGIFPPSVNMVIKDRPSDGLGVPSSSSGMASEISSELSTSEMSSEVGSTASDEPPPGALSENSPAYPSEQRCMLHPICLSNSFQRQLSSATFSSAFSDNGLDSDDEEPIEGVFTNGSRVEVEVDIHCSRAKEKEKQQHLLQVPAEASDEGIVISANEDEPGLPRKADFSAVGTIGRRRANGSVAPQERSHNVIEVATDAPLRKPGGYFAAPAQPDPDDQFIIPPELEEEVKEIMKHHQEQQQQQQPPPPPQLQPQLPRHYQLDQLPDYYDTPL",
# PHLP2_HUMAN
"MKRNGSRNCLNRRSRFGSRERDWLREDVKRGCVYLYGADTTTATTTTTTSSSSSSSSSSSDLHLVLCTVETPASEICAGEGRESLYLQLHGDLVRRLEPTERPLQIVYDYLSRLGFDDPVRIQEEATNPDLGCMIRFYGEKPCHMDRLDRILLSGIYNVRKGKTQLHKWAERLVVLCGTCLIVSSVKDCQTGKMHILPLVGGKIEEVKRRQYSLAFSSAGAQAQTYHVSFETLAEYQRWQRQASKVVSQRISTVDLSCYSLEEVPEHLFYSQDITYLNLRHNFMQLERPGGLDTLYKFSQLKGLNLSHNKLGLFPILLCEISTLTELNLSCNGFHDLPSQIGNLLNLQTLCLDGNFLTTLPEELGNLQQLSSLGISFNNFSQIPEVYEKLTMLDRVVMAGNCLEVLNLGVLNRMNHIKHVDLRMNHLKTMVIENLEGNKHITHVDLRDNRLTDLDLSSLCSLEQLHCGRNQLRELTLSGFSLRTLYASSNRLTAVNVYPVPSLLTFLDLSRNLLECVPDWACEAKKIEVLDVSYNLLTEVPVRILSSLSLRKLMLGHNHVQNLPTLVEHIPLEVLDLQHNALTRLPDTLFSKALNLRYLNASANSLESLPSACTGEESLSMLQLLYLTNNLLTDQCIPVLVGHLHLRILHLANNQLQTFPASKLNKLEQLEELNLSGNKLKTIPTTIANCKRLHTLVAHSNNISIFPEILQLPQIQFVDLSCNDLTEILIPEALPATLQDLDLTGNTNLVLEHKTLDIFSHITTLKIDQKPLPTTDSTVTSTFWSHGLAEMAGQRNKLCVSALAMDSFAEGVGAVYGMFDGDRNEELPRLLQCTMADVLLEEVQQSTNDTVFMANTFLVSHRKLGMAGQKLGSSALLCYIRPDTADPASSFSLTVANVGTCQAVLCRGGKPVPLSKVFSLEQDPEEAQRVKDQKAIITEDNKVNGVTCCTRMLGCTYLYPWILPKPHISSTPLTIQDELLILGNKALWEHLSYTEAVNAVRHVQDPLAAAKKLCTLAQSYGCQDNVGAMVVYLNIGEEGCTCEMNGLTLPGPVGFASTTTIKDAPKPATPSSSSGIASEFSSEMSTSEVSSEVGSTASDEHNAGGLDTALLPRPERRCSLHPTPTSGLFQRQPSSATFSSNQSDNGLDSDDDQPVEGVITNGSKVEVEVDIHCCRGRDLENSPPLIESSPTLCSEEHARGSCFGIRRQNSVNSGMLLPMSKDRMELQKSPSTSCLYGKKLSNGSIVPLEDSLNLIEVATEVPKRKTGYFAAPTQMEPEDQFVVPHDLEEEVKEQMKQHQDSRLEPEPHEEDRTEPPEEFDTAL",
# PP2D1_HUMAN
"MSTNNALRVFWKSREWNMKTSTFDSDEDILLLPKRKRFRKKKSRPVRHTKRHEEEQVYEQGTTLPCSICKHEIDLTGIFLHKKQHVALATLGFQWMGRKKPQPSVIAVQRQFMISKLLSSFMFTEKTLQSINNAFELLWKKQIPAYYKIFDNIDRSVIYSQKICHLLIKGVGICEDRNSTWKADMNDKFTVVSNFGNKPNVCFFGLFDGHHGASAAELTSMELPVLLLHQLSKFDPSYQMTTDEQQIINSFYTVFREEYAAIEDLFSAINKTEAVRCEYEDTHKAFAKAFWRMDRLLGLGRKEVSRVQWSGCSAVTCILEGKPKSPYAHKNWKRKNTHDGLAESSPSQEMPKIISGILHVANTGNVQAVLCRNGKGFCLTKEHTTRNTNERRRILQNGAVISSNEPYGLVEGQVKTTRGLGFHGNLKLKKSIIPAPQTISVPIDDLCQFLIVATNGLWEVLDKEEVTALAMTTFHMYKETYCPIIPNKSPSKGPLLFSTSEPNLTKSQSNIHVLFQYKSVSEVRVSTTNSKENLSDSNYSKYCIYNPENVETFPAETTHRKPCSEKVTDRPTSVNDVATNEKESDTKSFYEGAAEYVSHELVNAALLAGSRDNITVMVIFLNGSEYQLLT",
# PPM1A_HUMAN
"MGAFLDKPKMEKHNAQGQGNGLRYGLSSMQGWRVEMEDAHTAVIGLPSGLESWSFFAVYDGHAGSQVAKYCCEHLLDHITNNQDFKGSAGAPSVENVKNGIRTGFLEIDEHMRVMSEKKHGADRSGSTAVGVLISPQHTYFINCGDSRGLLCRNRKVHFFTQDHKPSNPLEKERIQNAGGSVMIQRVNGSLAVSRALGDFDYKCVHGKGPTEQLVSPEPEVHDIERSEEDDQFIILACDGIWDVMGNEELCDFVRSRLEVTDDLEKVCNEVVDTCLYKGSRDNMSVILICFPNAPKVSPEAVKKEAELDKYLECRVEEIIKKQGEGVPDLVHVMRTLASENIPSLPPGGELASKRNVIEAVYNRLNPYKNDDTDSTSTDDMW",
# PPM1B_HUMAN
"MGAFLDKPKTEKHNAHGAGNGLRYGLSSMQGWRVEMEDAHTAVVGIPHGLEDWSFFAVYDGHAGSRVANYCSTHLLEHITTNEDFRAAGKSGSALELSVENVKNGIRTGFLKIDEYMRNFSDLRNGMDRSGSTAVGVMISPKHIYFINCGDSRAVLYRNGQVCFSTQDHKPCNPREKERIQNAGGSVMIQRVNGSLAVSRALGDYDYKCVDGKGPTEQLVSPEPEVYEILRAEEDEFIILACDGIWDVMSNEELCEYVKSRLEVSDDLENVCNWVVDTCLHKGSRDNMSIVLVCFSNAPKVSDEAVKKDSELDKHLESRVEEIMEKSGEEGMPDLAHVMRILSAENIPNLPPGGGLAGKRNVIEAVYSRLNPHRESDGASDEAEESGSQGKLVEALRQMRINHRGNYRQLLEEMLTSYRLAKVEGEESPAEPAATATSSNSDAGNPVTMQESHTESESGLAELDSSNEDAGTKMSGEKI",
# PPM1N_HUMAN
"MAVLARQLQRLLWTACKKKEREKEGREEEEEEEAGRRAPEGPRSLLTAPRRAQRPHGGAEASGGLRFGASAAQGWRARMEDAHCTWLSLPGLPPGWALFAVLDGHGGARAARFGARHLPGHVLQELGPEPSEPEGVREALRRAFLSADERLRSLWPRVETGGCTAVVLLVSPRFLYLAHCGDSRAVLSRAGAVAFSTEDHRPLRPRERERIHAAGGTIRRRRVEGSLAVSRALGDFTYKEAPGRPPELQLVSAEPEVAALARQAEDEFMLLASDGVWDTVSGAALAGLVASRLRLGLAPELLCAQLLDTCLCKGSLDNMTCILVCFPGAPRPSEEAIRRELALDAALGCRIAELCASAQKPPSLNTVFRTLASEDIPDLPPGGGLDCKATVIAEVYSQICQVSEECGEKGQDGAGKSNPTHLGSALDMEA",
# PPM1D_HUMAN
"MAGLYSLGVSVFSDQGGRKYMEDVTQIVVEPEPTAEEKPSPRRSLSQPLPPRPSPAALPGGEVSGKGPAVAAREARDPLPDAGASPAPSRCCRRRSSVAFFAVCDGHGGREAAQFAREHLWGFIKKQKGFTSSEPAKVCAAIRKGFLACHLAMWKKLAEWPKTMTGLPSTSGTTASVVIIRGMKMYVAHVGDSGVVLGIQDDPKDDFVRAVEVTQDHKPELPKERERIEGLGGSVMNKSGVNRVVWKRPRLTHNGPVRRSTVIDQIPFLAVARALGDLWSYDFFSGEFVVSPEPDTSVHTLDPQKHKYIILGSDGLWNMIPPQDAISMCQDQEEKKYLMGEHGQSCAKMLVNRALGRWRQRMLRADNTSAIVICISPEVDNQGNFTNEDELYLNLTDSPSYNSQETCVMTPSPCSTPPVKSLEEDPWPRVNSKDHIPALVRSNAFSENFLEVSAEIARENVQGVVIPSKDPEPLEENCAKALTLRIHDSLNNSLPIGLVPTNSTNTVMDQKNLKMSTPGQMKAQEIERTPPTNFKRTLEESNSGPLMKKHRRNGLSRSSGAQPASLPTTSQRKNSVKLTMRRRLRGQKKIGNPLLHQHRKTVCVC",
# PPM1E_HUMAN
"MAGCIPEEKTYRRFLELFLGEFRGPCGGGEPEPEPEPEPEPEPESEPEPEPELVEAEAAEASVEEPGEEAATVAATEEGDQEQDPEPEEEAAVEGEEEEEGAATAAAAPGHSAVPPPPPQLPPLPPLPRPLSERITREEVEGESLDLCLQQLYKYNCPSFLAAALARATSDEVLQSDLSAHYIPKETDGTEGTVEIETVKLARSVFSKLHEICCSWVKDFPLRRRPQLYYETSIHAIKNMRRKMEDKHVCIPDFNMLFNLEDQEEQAYFAVFDGHGGVDAAIYASIHLHVNLVRQEMFPHDPAEALCRAFRVTDERFVQKAARESLRCGTTGVVTFIRGNMLHVAWVGDSQVMLVRKGQAVELMKPHKPDREDEKQRIEALGGCVVWFGAWRVNGSLSVSRAIGDAEHKPYICGDADSASTVLDGTEDYLILACDGFYDTVNPDEAVKVVSDHLKENNGDSSMVAHKLVASARDAGSSDNITVIVVFLRDMNKAVNVSEESDWTENSFQGGQEDGGDDKENHGECKRPWPQHQCSAPADLGYDGRVDSFTDRTSLSPGSQINVLEDPGYLDLTQIEASKPHSAQFLLPVEMFGPGAPKKANLINELMMEKKSVQSSLPEWSGAGEFPTAFNLGSTGEQIYRMQSLSPVCSGLENEQFKSPGNRVSRLSHLRHHYSKKWHRFRFNPKFYSFLSAQEPSHKIGTSLSSLTGSGKRNRIRSSLPWRQNSWKGYSENMRKLRKTHDIPCPDLPWSYKIE",
# PPM1M_HUMAN
"MSAGWFRRRFLPGEPLPAPRPPGPHASPVPYRRPRFLRGSSSSPGAADASRRPDSRPVRSPARGRTLPWNAGYAEIINAEKSEFNEDQAACGKLCIRRCEFGAEEEWLTLCPEEFLTGHYWALFDGHGGPAAAILAANTLHSCLRRQLEAVVEGLVATQPPMHLNGRCICPSDPQFVEEKGIRAEDLVIGALESAFQECDEVIGRELEASGQMGGCTALVAVSLQGKLYMANAGDSRAILVRRDEIRPLSFEFTPETERQRIQQLAFVYPELLAGEFTRLEFPRRLKGDDLGQKVLFRDHHMSGWSYKRVEKSDLKYPLIHGQGRQARLLGTLAVSRGLGDHQLRVLDTNIQLKPFLLSVPQVTVLDVDQLELQEDDVVVMATDGLWDVLSNEQVAWLVRSFLPGNQEDPHRFSKLAQMLIHSTQGKEDSLTEEGQVSYDDVSVFVIPLHSQGQESSDH",
# PPM1F_HUMAN
"MSSGAPQKSSPMASGAEETPGFLDTLLQDFPALLNPEDPLPWKAPGTVLSQEEVEGELAELAMGFLGSRKAPPPLAAALAHEAVSQLLQTDLSEFRKLPREEEEEEEDDDEEEKAPVTLLDAQSLAQSFFNRLWEVAGQWQKQVPLAARASQRQWLVSIHAIRNTRRKMEDRHVSLPSFNQLFGLSDPVNRAYFAVFDGHGGVDAARYAAVHVHTNAARQPELPTDPEGALREAFRRTDQMFLRKAKRERLQSGTTGVCALIAGATLHVAWLGDSQVILVQQGQVVKLMEPHRPERQDEKARIEALGGFVSHMDCWRVNGTLAVSRAIGDVFQKPYVSGEADAASRALTGSEDYLLLACDGFFDVVPHQEVVGLVQSHLTRQQGSGLRVAEELVAAARERGSHDNITVMVVFLRDPQELLEGGNQGEGDPQAEGRRQDLPSSLPEPETQAPPRS",
# PPM1G_HUMAN
"MGAYLSQPNTVKCSGDGVGAPRLPLPYGFSAMQGWRVSMEDAHNCIPELDSETAMFSVYDGHGGEEVALYCAKYLPDIIKDQKAYKEGKLQKALEDAFLAIDAKLTTEEVIKELAQIAGRPTEDEDEKEKVADEDDVDNEEAALLHEEATMTIEELLTRYGQNCHKGPPHSKSGGGTGEEPGSQGLNGEAGPEDSTRETPSQENGPTAKAYTGFSSNSERGTEAGQVGEPGIPTGEAGPSCSSASDKLPRVAKSKFFEDSEDESDEAEEEEEDSEECSEEEDGYSSEEAENEEDEDDTEEAEEDDEEEEEEMMVPGMEGKEEPGSDSGTTAVVALIRGKQLIVANAGDSRCVVSEAGKALDMSYDHKPEDEVELARIKNAGGKVTMDGRVNGGLNLSRAIGDHFYKRNKNLPPEEQMISALPDIKVLTLTDDHEFMVIACDGIWNVMSSQEVVDFIQSKISQRDENGELRLLSSIVEELLDQCLAPDTSGDGTGCDNMTCIIICFKPRNTAELQPESGKRKLEEVLSTEGAEENGNSDKKKKAKRD",
# PPM1H_HUMAN
"MLTRVKSAVANFMGGIMAGSSGSEHGGGSCGGSDLPLRFPYGRPEFLGLSQDEVECSADHIARPILILKETRRLPWATGYAEVINAGKSTHNEDQASCEVLTVKKKAGAVTSTPNRNSSKRRSSLPNGEGLQLKENSESEGVSCHYWSLFDGHAGSGAAVVASRLLQHHITEQLQDIVDILKNSAVLPPTCLGEEPENTPANSRTLTRAASLRGGVGAPGSPSTPPTRFFTEKKIPHECLVIGALESAFKEMDLQIERERSSYNISGGCTALIVICLLGKLYVANAGDSRAIIIRNGEIIPMSSEFTPETERQRLQYLAFMQPHLLGNEFTHLEFPRRVQRKELGKKMLYRDFNMTGWAYKTIEDEDLKFPLIYGEGKKARVMATIGVTRGLGDHDLKVHDSNIYIKPFLSSAPEVRIYDLSKYDHGSDDVLILATDGLWDVLSNEEVAEAITQFLPNCDPDDPHRYTLAAQDLVMRARGVLKDRGWRISNDRLGSGDDISVYVIPLIHGNKLS",
# PPM1J_HUMAN
"MLNRVRSAVAHLVSSGGAPPPRPKSPDLPNAASAPPAAAPEAPRSPPAKAGSGSATPAKAVEARASFSRPTFLQLSPGGLRRADDHAGRAVQSPPDTGRRLPWSTGYAEVINAGKSRHNEDQACCEVVYVEGRRSVTGVPREPSRGQGLCFYYWGLFDGHAGGGAAEMASRLLHRHIREQLKDLVEILQDPSPPPLCLPTTPGTPDSSDPSHLLGPQSCWSSQKEVSHESLVVGAVENAFQLMDEQMARERRGHQVEGGCCALVVIYLLGKVYVANAGDSRAIIVRNGEIIPMSREFTPETERQRLQLLGFLKPELLGSEFTHLEFPRRVLPKELGQRMLYRDQNMTGWAYKKIELEDLRFPLVCGEGKKARVMATIGVTRGLGDHSLKVCSSTLPIKPFLSCFPEVRVYDLTQYEHCPDDVLVLGTDGLWDVTTDCEVAATVDRVLSAYEPNDHSRYTALAQALVLGARGTPRDRGWRLPNNKLGSGDDISVFVIPLGGPGSYS",
# PPM1K_HUMAN
"MSTAALITLVRSGGNQVRRRVLLSSRLLQDDRRVTPTCHSSTSEPRCSRFDPDGSGSPATWDNFGIWDNRIDEPILLPPSIKYGKPIPKISLENVGCASQIGKRKENEDRFDFAQLTDEVLYFAVYDGHGGPAAADFCHTHMEKCIMDLLPKEKNLETLLTLAFLEIDKAFSSHARLSADATLLTSGTTATVALLRDGIELVVASVGDSRAILCRKGKPMKLTIDHTPERKDEKERIKKCGGFVAWNSLGQPHVNGRLAMTRSIGDLDLKTSGVIAEPETKRIKLHHADDSFLVLTTDGINFMVNSQEICDFVNQCHDPNEAAHAVTEQAIQYGTEDNSTAVVVPFGAWGKYKNSEINFSFSRSFASSGRWA",
# PPM1L_HUMAN
"MIEDTMTLLSLLGRIMRYFLLRPETLFLLCISLALWSYFFHTDEVKTIVKSSRDAVKMVKGKVAEIMQNDRLGGLDVLEAEFSKTWEFKNHNVAVYSIQGRRDHMEDRFEVLTDLANKTHPSIFGIFDGHGGETAAEYVKSRLPEALKQHLQDYEKDKENSVLSYQTILEQQILSIDREMLEKLTVSYDEAGTTCLIALLSDKDLTVANVGDSRGVLCDKDGNAIPLSHDHKPYQLKERKRIKRAGGFISFNGSWRVQGILAMSRSLGDYPLKNLNVVIPDPDILTFDLDKLQPEFMILASDGLWDAFSNEEAVRFIKERLDEPHFGAKSIVLQSFYRGCPDNITVMVVKFRNSSKTEEQ",
# PPTC7_HUMAN
"MFSVLSYGRLVARAVLGGLSQTDPRAGGGGGGDYGLVTAGCGFGKDFRKGLLKKGACYGDDACFVARHRSADVLGVADGVGGWRDYGVDPSQFSGTLMRTCERLVKEGRFVPSNPIGILTTSYCELLQNKVPLLGSSTACIVVLDRTSHRLHTANLGDSGFLVVRGGEVVHRSDEQQHYFNTPFQLSIAPPEAEGVVLSDSPDAADSTSFDVQLGDIILTATDGLFDNMPDYMILQELKKLKNSNYESIQQTARSIAEQAHELAYDPNYMSPFAQFACDNGLNVRGGKPDDITVLLSIVAEYTD",
# TAB1_HUMAN
"MAAQRRSLLQSEQQPSWTDDLPLCHLSGVGSASNRSYSADGKGTESHPPEDSWLKFRSENNCFLYGVFNGYDGNRVTNFVAQRLSAELLLGQLNAEHAEADVRRVLLQAFDVVERSFLESIDDALAEKASLQSQLPEGVPQHQLPPQYQKILERLKTLEREISGGAMAVVAVLLNNKLYVANVGTNRALLCKSTVDGLQVTQLNVDHTTENEDELFRLSQLGLDAGKIKQVGIICGQESTRRIGDYKVKYGYTDIDLLSAAKSKPIIAEPEIHGAQPLDGVTGFLVLMSEGLYKALEAAHGPGQANQEIAAMIDTEFAKQTSLDAVAQAVVDRVKRIHSDTFASGGERARFCPRHEDMTLLVRNFGYPLGEMSQPTPSPAPAAGGRVYPVSVPYSSAQSTSKTSVTLSLVMPSQGQMVNGAHSASTLDEATPTLTNQSPTLTLQSTNTHTQSSSSSSDGGLFRSRPAHSLPPGEDGRVEPYVDFAEFYRLWSVDHGEQSVVTAP",
# PPA5_HUMAN
"MDMWTALLILQALLLPSLADGATPALRFVAVGDWGGVPNAPFHTAREMANAKEIARTVQILGADFILSLGDNFYFTGVQDINDKRFQETFEDVFSDRSLRKVPWYVLAGNHDHLGNVSAQIAYSKISKRWNFPSPFYRLHFKIPQTNVSVAIFMLDTVTLCGNSDDFLSQQPERPRDVKLARTQLSWLKKQLAAAREDYVLVAGHYPVWSIAEHGPTHCLVKQLRPLLATYGVTAYLCGHDHNLQYLQDENGVGYVLSGAGNFMDPSKRHQRKVPNGYLRFHYGTEDSLGGFAYVEISSKEMTVTYIEASGKSLFKTRLPRRARP",
# ACP7_HUMAN
"MHPLPGYWSCYCLLLLFSLGVQGSLGAPSAAPEQVHLSYPGEPGSMTVTWTTWVPTRSEVQFGLQPSGPLPLRAQGTFVPFVDGGILRRKLYIHRVTLRKLLPGVQYVYRCGSAQGWSRRFRFRALKNGAHWSPRLAVFGDLGADNPKAVPRLRRDTQQGMYDAVLHVGDFAYNLDQDNARVGDRFMRLIEPVAASLPYMTCPGNHEERYNFSNYKARFSMPGDNEGLWYSWDLGPAHIISFSTEVYFFLHYGRHLVQRQFRWLESDLQKANKNRAARPWIITMGHRPMYCSNADLDDCTRHESKVRKGLQGKLYGLEDLFYKYGVDLQLWAHEHSYERLWPIYNYQVFNGSREMPYTNPRGPVHIITGSAGCEERLTPFAVFPRPWSAVRVKEYGYTRLHILNGTHIHIQQVSDDQDGKIVDDVWVVRPLFGRRMYL",
# PP1A_HUMAN
"MSDSEKLNLDSIIGRLLEVQGSRPGKNVQLTENEIRGLCLKSREIFLSQPILLELEAPLKICGDIHGQYYDLLRLFEYGGFPPESNYLFLGDYVDRGKQSLETICLLLAYKIKYPENFFLLRGNHECASINRIYGFYDECKRRYNIKLWKTFTDCFNCLPIAAIVDEKIFCCHGGLSPDLQSMEQIRRIMRPTDVPDQGLLCDLLWSDPDKDVQGWGENDRGVSFTFGAEVVAKFLHKHDLDLICRAHQVVEDGYEFFAKRQLVTLFSAPNYCGEFDNAGAMMSVDETLMCSFQILKPADKNKGKYGQFSGLNPGGRPITPPRNSAKAKK",
# PP1B_HUMAN
"MADGELNVDSLITRLLEVRGCRPGKIVQMTEAEVRGLCIKSREIFLSQPILLELEAPLKICGDIHGQYTDLLRLFEYGGFPPEANYLFLGDYVDRGKQSLETICLLLAYKIKYPENFFLLRGNHECASINRIYGFYDECKRRFNIKLWKTFTDCFNCLPIAAIVDEKIFCCHGGLSPDLQSMEQIRRIMRPTDVPDTGLLCDLLWSDPDKDVQGWGENDRGVSFTFGADVVSKFLNRHDLDLICRAHQVVEDGYEFFAKRQLVTLFSAPNYCGEFDNAGGMMSVDETLMCSFQILKPSEKKAKYQYGGLNSGRPVTPPRTANPPKKR",
# PP1G_HUMAN
"MADLDKLNIDSIIQRLLEVRGSKPGKNVQLQENEIRGLCLKSREIFLSQPILLELEAPLKICGDIHGQYYDLLRLFEYGGFPPESNYLFLGDYVDRGKQSLETICLLLAYKIKYPENFFLLRGNHECASINRIYGFYDECKRRYNIKLWKTFTDCFNCLPIAAIVDEKIFCCHGGLSPDLQSMEQIRRIMRPTDVPDQGLLCDLLWSDPDKDVLGWGENDRGVSFTFGAEVVAKFLHKHDLDLICRAHQVVEDGYEFFAKRQLVTLFSAPNYCGEFDNAGAMMSVDETLMCSFQILKPAEKKKPNATRPVTPPRGMITKQAKK",
# PP2AA_HUMAN
"MDEKVFTKELDQWIEQLNECKQLSESQVKSLCEKAKEILTKESNVQEVRCPVTVCGDVHGQFHDLMELFRIGGKSPDTNYLFMGDYVDRGYYSVETVTLLVALKVRYRERITILRGNHESRQITQVYGFYDECLRKYGNANVWKYFTDLFDYLPLTALVDGQIFCLHGGLSPSIDTLDHIRALDRLQEVPHEGPMCDLLWSDPDDRGGWGISPRGAGYTFGQDISETFNHANGLTLVSRAHQLVMEGYNWCHDRNVVTIFSAPNYCYRCGNQAAIMELDDTLKYSFLQFDPAPRRGEPHVTRRTPDYFL",
# PP2AB_HUMAN
"MDDKAFTKELDQWVEQLNECKQLNENQVRTLCEKAKEILTKESNVQEVRCPVTVCGDVHGQFHDLMELFRIGGKSPDTNYLFMGDYVDRGYYSVETVTLLVALKVRYPERITILRGNHESRQITQVYGFYDECLRKYGNANVWKYFTDLFDYLPLTALVDGQIFCLHGGLSPSIDTLDHIRALDRLQEVPHEGPMCDLLWSDPDDRGGWGISPRGAGYTFGQDISETFNHANGLTLVSRAHQLVMEGYNWCHDRNVVTIFSAPNYCYRCGNQAAIMELDDTLKYSFLQFDPAPRRGEPHVTRRTPDYFL",
# PP2BA_HUMAN
"MSEPKAIDPKLSTTDRVVKAVPFPPSHRLTAKEVFDNDGKPRVDILKAHLMKEGRLEESVALRIITEGASILRQEKNLLDIDAPVTVCGDIHGQFFDLMKLFEVGGSPANTRYLFLGDYVDRGYFSIECVLYLWALKILYPKTLFLLRGNHECRHLTEYFTFKQECKIKYSERVYDACMDAFDCLPLAALMNQQFLCVHGGLSPEINTLDDIRKLDRFKEPPAYGPMCDILWSDPLEDFGNEKTQEHFTHNTVRGCSYFYSYPAVCEFLQHNNLLSILRAHEAQDAGYRMYRKSQTTGFPSLITIFSAPNYLDVYNNKAAVLKYENNVMNIRQFNCSPHPYWLPNFMDVFTWSLPFVGEKVTEMLVNVLNICSDDELGSEEDGFDGATAAARKEVIRNKIRAIGKMARVFSVLREESESVLTLKGLTPTGMLPSGVLSGGKQTLQSATVEAIEADEAIKGFSPQHKITSFEEAKGLDRINERMPPRRDAMPSDANLNSINKALTSETNGTDSNGSNSSNIQ",
# PP2BB_HUMAN
"MAAPEPARAAPPPPPPPPPPPGADRVVKAVPFPPTHRLTSEEVFDLDGIPRVDVLKNHLVKEGRVDEEIALRIINEGAAILRREKTMIEVEAPITVCGDIHGQFFDLMKLFEVGGSPANTRYLFLGDYVDRGYFSIECVLYLWVLKILYPSTLFLLRGNHECRHLTEYFTFKQECKIKYSERVYEACMEAFDSLPLAALLNQQFLCVHGGLSPEIHTLDDIRRLDRFKEPPAFGPMCDLLWSDPSEDFGNEKSQEHFSHNTVRGCSYFYNYPAVCEFLQNNNLLSIIRAHEAQDAGYRMYRKSQTTGFPSLITIFSAPNYLDVYNNKAAVLKYENNVMNIRQFNCSPHPYWLPNFMDVFTWSLPFVGEKVTEMLVNVLSICSDDELMTEGEDQFDGSAAARKEIIRNKIRAIGKMARVFSVLREESESVLTLKGLTPTGMLPSGVLAGGRQTLQSATVEAIEAEKAIRGFSPPHRICSFEEAKGLDRINERMPPRKDAVQQDGFNSLNTAHATENHGTGNHTAQ",
# PP2BC_HUMAN
"MSGRRFHLSTTDRVIKAVPFPPTQRLTFKEVFENGKPKVDVLKNHLVKEGRLEEEVALKIINDGAAILRQEKTMIEVDAPITVCGDIHGQFFDLMKLFEVGGSPSNTRYLFLGDYVDRGYFSIECVLYLWSLKINHPKTLFLLRGNHECRHLTDYFTFKQECRIKYSEQVYDACMETFDCLPLAALLNQQFLCVHGGMSPEITSLDDIRKLDRFTEPPAFGPVCDLLWSDPSEDYGNEKTLEHYTHNTVRGCSYFYSYPAVCEFLQNNNLLSIIRAHEAQDAGYRMYRKSQATGFPSLITIFSAPNYLDVYNNKAAVLKYENNVMNIRQFNCSPHPYWLPNFMDVFTWSLPFVGEKVTEMLVNVLNICSDDELISDDEAEGSTTVRKEIIRNKIRAIGKMARVFSILRQESESVLTLKGLTPTGTLPLGVLSGGKQTIETATVEAVEAREAIRGFSLQHKIRSFEEARGLDRINERMPPRKDSIHAGGPMKSVTSAHSHAAHRSDQGKKAHS",
# PP4C_HUMAN
"MAEISDLDRQIEQLRRCELIKESEVKALCAKAREILVEESNVQRVDSPVTVCGDIHGQFYDLKELFRVGGDVPETNYLFMGDFVDRGFYSVETFLLLLALKVRYPDRITLIRGNHESRQITQVYGFYDECLRKYGSVTVWRYCTEIFDYLSLSAIIDGKIFCVHGGLSPSIQTLDQIRTIDRKQEVPHDGPMCDLLWSDPEDTTGWGVSPRGAGYLFGSDVVAQFNAANDIDMICRAHQLVMEGYKWHFNETVLTVWSAPNYCYRCGNVAAILELDEHLQKDFIIFEAAPQETRGIPSKKPVADYFL",
# PPP5_HUMAN
"MAMAEGERTECAEPPRDEPPADGALKRAEELKTQANDYFKAKDYENAIKFYSQAIELNPSNAIYYGNRSLAYLRTECYGYALGDATRAIELDKKYIKGYYRRAASNMALGKFRAALRDYETVVKVKPHDKDAKMKYQECNKIVKQKAFERAIAGDEHKRSVVDSLDIESMTIEDEYSGPKLEDGKVTISFMKELMQWYKDQKKLHRKCAYQILVQVKEVLSKLSTLVETTLKETEKITVCGDTHGQFYDLLNIFELNGLPSETNPYIFNGDFVDRGSFSVEVILTLFGFKLLYPDHFHLLRGNHETDNMNQIYGFEGEVKAKYTAQMYELFSEVFEWLPLAQCINGKVLIMHGGLFSEDGVTLDDIRKIERNRQPPDSGPMCDLLWSDPQPQNGRSISKRGVSCQFGPDVTKAFLEENNLDYIIRSHEVKAEGYEVAHGGRCVTVFSAPNYCDQMGNKASYIHLQGSDLRPQFHQFTAVPHPNVKPMAYANTLLQLGMM",
# PPP6_HUMAN
"MAPLDLDKYVEIARLCKYLPENDLKRLCDYVCDLLLEESNVQPVSTPVTVCGDIHGQFYDLCELFRTGGQVPDTNYIFMGDFVDRGYYSLETFTYLLALKAKWPDRITLLRGNHESRQITQVYGFYDECQTKYGNANAWRYCTKVFDMLTVAALIDEQILCVHGGLSPDIKTLDQIRTIERNQEIPHKGAFCDLVWSDPEDVDTWAISPRGAGWLFGAKVTNEFVHINNLKLICRAHQLVHEGYKFMFDEKLVTVWSAPNYCYRCGNIASIMVFKDVNTREPKLFRAVPDSERVIPPRTTTPYFL",
# PPE1_HUMAN
"MGCSSSSTKTRRSDTSLRAALIIQNWYRGYKARLKARQHYALTIFQSIEYADEQGQMQLSTFFSFMLENYTHIHKEELELRNQSLESEQDMRDRWDYVDSIDVPDSYNGPRLQFPLTCTDIDLLLEAFKEQQILHAHYVLEVLFETKKVLKQMPNFTHIQTSPSKEVTICGDLHGKLDDLFLIFYKNGLPSERNPYVFNGDFVDRGKNSIEILMILCVSFLVYPNDLHLNRGNHEDFMMNLRYGFTKEILHKYKLHGKRILQILEEFYAWLPIGTIVDNEILVIHGGISETTDLNLLHRVERNKMKSVLIPPTETNRDHDTDSKHNKVGVTFNAHGRIKTNGSPTEHLTEHEWEQIIDILWSDPRGKNGCFPNTCRGGGCYFGPDVTSKILNKYQLKMLIRSHECKPEGYEICHDGKVVTIFSASNYYEEGSNRGAYIKLCSGTTPRFFQYQVTKATCFQPLRQRVDTMENSAIKILRERVISRKSDLTRAFQLQDHRKSGKLSVSQWAFCMENILGLNLPWRSLSSNLVNIDQNGNVEYMSSFQNIRIEKPVQEAHSTLVETLYRYRSDLEIIFNAIDTDHSGLISVEEFRAMWKLFSSHYNVHIDDSQVNKLANIMDLNKDGSIDFNEFLKAFYVVHRYEDLMKPDVTNLG",
# PPE2_HUMAN
"MGSGTSTQHHFAFQNAERAFKAAALIQRWYRRYVARLEMRRRCTWSIFQSIEYAGQQDQVKLHDFFSYLMDHFIPSSHNDRDFLTRIFTEDRFAQDSEMKKCSDYESIEVPDSYTGPRLSFPLLPDHATALVEAFRLKQQLHARYVLNLLYETKKHLVQLPNINRVSTCYSEEITVCGDLHGQLDDLIFIFYKNGLPSPERSYVFNGDFVDRGKDSVEILMILFAFMLVYPKEFHLNRGNHEDHMVNLRYGFTKEVMNKYKVHGKEILRTLQDVFCWLPLATLIDEKVLILHGGVSDITDLELLDKIERSKIVSTMRCKTRQKSEKQMEEKRRANQKSSAQGPIPWFLPESRSLPSSPLRLGSYKAQKTSRSSSIPCSGSLDGRELSRQVRSSVELELERCRQQAGLLVTGEKEEPSRSASEADSEAGELRKPTQEEWRQVVDILWSDPMAQEGCKANTIRGGGCYFGPDVTQQLLQKYNMQFLIRSHECKPEGYEFCHNRKVLTIFSASNYYEVGSNRGAYVKLGPALTPHIVQYQANKVTHTLTMRQRISRVEESALRALREKLFAHSSDLLSEFKKHDADKVGLITLSDWAAAVESVLHLGLPWRMLRPQLVNSSADNMLEYKSWLKNLAKEQLSRENIQSSLLETLYRNRSNLETIFRIIDSDHSGFISLDEFRQTWKLFSSHMNIDITDDCICDLARSIDFNKDGHIDINEFLEAFRLVEKSCPEGDASECPQATNAKDSGCSSPGAH",
# RPAP2_HUMAN
"MADFAGPSSAGRKAGAPRCSRKAAGTKQTSTLKQEDASKRKAELEAAVRKKIEFERKALHIVEQLLEENITEEFLMECGRFITPAHYSDVVDERSIVKLCGYPLCQKKLGIVPKQKYKISTKTNKVYDITERKSFCSNFCYQASKFFEAQIPKTPVWVREEERHPDFQLLKEEQSGHSGEEVQLCSKAIKTSDIDNPSHFEKQYESSSSSTHSDSSSDNEQDFVSSILPGNRPNSTNIRPQLHQKSIMKKKAGHKANSKHKDKEQTVVDVTEQLGDCKLDSQEKDATCELPLQKVNTQSSSNSTLPERLKASENSESEYSRSEITLVGISKKSAEHFKRKFAKSNQVSRSVSSSVQVCPEVGKRNLLKVLKETLIEWKTEETLRFLYGQNYASVCLKPEASLVKEELDEDDIISDPDSHFPAWRESQNSLDESLPFRGSGTAIKPLPSYENLKKETEKLNLRIREFYRGRYVLGEETTKSQDSEEHDSTFPLIDSSSQNQIRKRIVLEKLSKVLPGLLVPLQITLGDIYTQLKNLVRTFRLTNRNIIHKPAEWTLIAMVLLSLLTPILGIQKHSQEGMVFTRFLDTLLEELHLKNEDLESLTIIFRTSCLPE",
# ITPA_HUMAN
"MAASLVGKKIVFVTGNAKKLEEVVQILGDKFPCTLVAQKIDLPEYQGEPDEISIQKCQEAVRQVQGPVLVEDTCLCFNALGGLPGPYIKWFLEKLKPEGLHQLLAGFEDKSAYALCTFALSTGDPSQPVRLFRGRTSGRIVAPRGCQDFGWDPCFQPDGYEQTYAEMPKAEKNAVSHRFRALLELQEYFGSLAA",
# DOPP1_HUMAN
"MAADGQCSLPASWRPVTLTHVEYPAGDLSGHLLAYLSLSPVFVIVGFVTLIIFKRELHTISFLGGLALNEGVNWLIKNVIQEPRPCGGPHTAVGTKYGMPSSHSQFMWFFSVYSFLFLYLRMHQTNNARFLDLLWRHVLSLGLLAVAFLVSYSRVYLLYHTWSQVLYGGIAGGLMAIAWFIFTQEVLTPLFPRIAAWPVSEFFLIRDTSLIPNVLWFEYTVTRAEARNRQRKLGTKLQ",
# G6PC1_HUMAN
"MEEGMNVLHDFGIQSTHYLQVNYQDSQDWFILVSVIADLRNAFYVLFPIWFHLQEAVGIKLLWVAVIGDWLNLVFKWILFGQRPYWWVLDTDYYSNTSVPLIKQFPVTCETGPGSPSGHAMGTAGVYYVMVTSTLSIFQGKIKPTYRFRCLNVILWLGFWAVQLNVCLSRIYLAAHFPHQVVAGVLSGIAVAETFSHIHSIYNASLKKYFLITFFLFSFAIGFYLLLKGLGVDLLWTLEKAQRWCEQPEWVHIDTTPFASLLKNLGTLFGLGLALNSSMYRESCKGKLSKWLPFRLSSIVASLVLLHVFDSLKPPSQVELVFYVLSFCKSAVVPLASVSVIPYCLAQVLGQPHKKSL",
# G6PC2_HUMAN
"MDFLHRNGVLIIQHLQKDYRAYYTFLNFMSNVGDPRNIFFIYFPLCFQFNQTVGTKMIWVAVIGDWLNLIFKWILFGHRPYWWVQETQIYPNHSSPCLEQFPTTCETGPGSPSGHAMGASCVWYVMVTAALSHTVCGMDKFSITLHRLTWSFLWSVFWLIQISVCISRVFIATHFPHQVILGVIGGMLVAEAFEHTPGIQTASLGTYLKTNLFLFLFAVGFYLLLRVLNIDLLWSVPIAKKWCANPDWIHIDTTPFAGLVRNLGVLFGLGFAINSEMFLLSCRGGNNYTLSFRLLCALTSLTILQLYHFLQIPTHEEHLFYVLSFCKSASIPLTVVAFIPYSVHMLMKQSGKKSQ",
# G6PC3_HUMAN
"MESTLGAGIVIAEALQNQLAWLENVWLWITFLGDPKILFLFYFPAAYYASRRVGIAVLWISLITEWLNLIFKWFLFGDRPFWWVHESGYYSQAPAQVHQFPSSCETGPGSPSGHCMITGAALWPIMTALSSQVATRARSRWVRVMPSLAYCTFLLAVGLSRIFILAHFPHQVLAGLITGAVLGWLMTPRVPMERELSFYGLTALALMLGTSLIYWTLFTLGLDLSWSISLAFKWCERPEWIHVDSRPFASLSRDSGAALGLGIALHSPCYAQVRRAQLGNGQKIACLVLAMGLLGPLDWLGHPPQISLFYIFNFLKYTLWPCLVLALVPWAVHMFSAQEAPPIHSS",
# PLPR1_HUMAN
"MAVGNNTQRSYSIIPCFIFVELVIMAGTVLLAYYFECTDTFQVHIQGFFCQDGDLMKPYPGTEEESFITPLVLYCVLAATPTAIIFIGEISMYFIKSTRESLIAQEKTILTGECCYLNPLLRRIIRFTGVFAFGLFATDIFVNAGQVVTGHLTPYFLTVCKPNYTSADCQAHHQFINNGNICTGDLEVIEKARRSFPSKHAALSIYSALYATMYITSTIKTKSSRLAKPVLCLGTLCTAFLTGLNRVSEYRNHCSDVIAGFILGTAVALFLGMCVVHNFKGTQGSPSKPKPEDPRGVPLMAFPRIESPLETLSAQNHSASMTEVT",
# PLPR2_HUMAN
"MAGGRPHLKRSFSIIPCFVFVESVLLGIVILLAYRLEFTDTFPVHTQGFFCYDSTYAKPYPGPEAASRVPPALVYALVTAGPTLTILLGELARAFFPAPPSAVPVIGESTIVSGACCRFSPPVRRLVRFLGVYSFGLFTTTIFANAGQVVTGNPTPHFLSVCRPNYTALGCLPPSPDRPGPDRFVTDQGACAGSPSLVAAARRAFPCKDAALCAYAVTYTAMYVTLVFRVKGSRLVKPSLCLALLCPAFLVGVVRVAEYRNHWSDVLAGFLTGAAIATFLVTCVVHNFQSRPPSGRRLSPWEDLGQAPTMDSPLEKNPRSAGRIRHRHGSPHPSRRTAPAVAT",
# PLPR3_HUMAN
"MISTKEKNKIPKDSMTLLPCFYFVELPIVASSIVSLYFLELTDLFKPAKVGFQCYDRTLSMPYVETNEELIPLLMLLSLAFAAPAASIMVAEGMLYCLQSRLWGRAGGPAGAEGSINAGGCNFNSFLRRTVRFVGVHVFGLCATALVTDVIQLATGYHTPFFLTVCKPNYTLLGTSCEVNPYITQDICSGHDIHAILSARKTFPSQHATLSAFAAVYVSMYFNSVISDTTKLLKPILVFAFAIAAGVCGLTQITQYRSHPVDVYAGFLIGAGIAAYLACHAVGNFQAPPAEKPAAPAPAKDALRALTQRGHDSVYQQNKSVSTDELGPPGRLEGAPRPVAREKTSLGSLKRASVDVDLLAPRSPMAKENMVTFSHTLPRASAPSLDDPARRHMTIHVPLDASRSKQLISEWKQKSLEGRGLGLPDDASPGHLRAPAEPMAEEEEEEEDEEEEEEEEEEEDEGPAPPSLYPTVQARPGLGPRVILPPRAGPPPLVHIPEEGAQTGAGLSPKSGAGVRAKWLMMAEKSGAAVANPPRLLQVIAMSKAPGAPGPKAAETASSSSASSDSSQYRSPSDRDSASIVTIDAHAPHHPVVHLSAGGAPWEWKAAGGGAKAEADGGYELGDLARGFRGGAKPPGVSPGSSVSDVDQEEPRFGAVATVNLATGEGLPPLGAADGALGPGSRESTLRRHAGGLGLAEREAEAEAEGYFRKMQARRFPD",
# PLPR4_HUMAN
"MQRAGSSGGRGECDISGAGRLGLEEAARLSCAVHTSPGGGRRPGQAAGMSAKERPKGKVIKDSVTLLPCFYFVELPILASSVVSLYFLELTDVFKPVHSGFSCYDRSLSMPYIEPTQEAIPFLMLLSLAFAGPAITIMVGEGILYCCLSKRRNGVGLEPNINAGGCNFNSFLRRAVRFVGVHVFGLCSTALITDIIQLSTGYQAPYFLTVCKPNYTSLNVSCKENSYIVEDICSGSDLTVINSGRKSFPSQHATLAAFAAVYVSMYFNSTLTDSSKLLKPLLVFTFIICGIICGLTRITQYKNHPVDVYCGFLIGGGIALYLGLYAVGNFLPSDESMFQHRDALRSLTDLNQDPNRLLSAKNGSSSDGIAHTEGILNRNHRDASSLTNLKRANADVEIITPRSPMGKENMVTFSNTLPRANTPSVEDPVRRNASIHASMDSARSKQLLTQWKNKNESRKLSLQVIEPEPGQSPPRSIEMRSSSEPSRVGVNGDHHGPGNQYLKIQPGAVPGCNNSMPGGPRVSIQSRPGSSQLVHIPEETQENISTSPKSSSARAKWLKAAEKTVACNRSNSQPRIMQVIAMSKQQGVLQSSPKNTEGSTVSCTGSIRYKTLTDHEPSGIVRVEAHPENNRPIIQIPSTEGEGSGSWKWKAPEKGSLRQTYELNDLNRDSESCESLKDSFGSGDRKRSNIDSNEHHHHGITTIRVTPVEGSEIGSETLSISSSRDSTLRRKGNIILIPERSNSPENTRNIFYKGTSPTRAYKD",
# PLPR5_HUMAN
"MPLLPAALTSSMLYFQMVIMAGTVMLAYYFEYTDTFTVNVQGFFCHDSAYRKPYPGPEDSSAVPPVLLYSLAAGVPVLVIIVGETAVFCLQLATRDFENQEKTILTGDCCYINPLVRRTVRFLGIYTFGLFATDIFVNAGQVVTGNLAPHFLALCKPNYTALGCQQYTQFISGEEACTGNPDLIMRARKTFPSKEAALSVYAAMYLTMYITNTIKAKGTRLAKPVLCLGLMCLAFLTGLNRVAEYRNHWSDVIAGFLVGISIAVFLVVCVVNNFKGRQAENEHIHMDNLAQMPMISIPRVESPLEKVTSVQNHITAFAEVT",
# PLPP1_HUMAN
"MFDKTRLPYVALDVLCVLLAGLPFAILTSRHTPFQRGVFCNDESIKYPYKEDTIPYALLGGIIIPFSIIVIILGETLSVYCNLLHSNSFIRNNYIATIYKAIGTFLFGAAASQSLTDIAKYSIGRLRPHFLDVCDPDWSKINCSDGYIEYYICRGNAERVKEGRLSFYSGHSSFSMYCMLFVALYLQARMKGDWARLLRPTLQFGLVAVSIYVGLSRVSDYKHHWSDVLTGLIQGALVAILVAVYVSDFFKERTSFKERKEEDSHTTLHETPTTGNHYPSNHQP",
# PLPP3_HUMAN
"MQNYKYDKAIVPESKNGGSPALNNNPRRSGSKRVLLICLDLFCLFMAGLPFLIIETSTIKPYHRGFYCNDESIKYPLKTGETINDAVLCAVGIVIAILAIITGEFYRIYYLKKSRSTIQNPYVAALYKQVGCFLFGCAISQSFTDIAKVSIGRLRPHFLSVCNPDFSQINCSEGYIQNYRCRGDDSKVQEARKSFFSGHASFSMYTMLYLVLYLQARFTWRGARLLRPLLQFTLIMMAFYTGLSRVSDHKHHPSDVLAGFAQGALVACCIVFFVSDLFKTKTTLSLPAPAIRKEILSPVDIIDRNNHHNMM",
# PLPP2_HUMAN
"MQRRWVFVLLDVLCLLVASLPFAILTLVNAPYKRGFYCGDDSIRYPYRPDTITHGLMAGVTITATVILVSAGEAYLVYTDRLYSRSDFNNYVAAVYKVLGTFLFGAAVSQSLTDLAKYMIGRLRPNFLAVCDPDWSRVNCSVYVQLEKVCRGNPADVTEARLSFYSGHSSFGMYCMVFLALYVQARLCWKWARLLRPTVQFFLVAFALYVGYTRVSDYKHHWSDVLVGLLQGALVAALTVCYISDFFKARPPQHCLKEEELERKPSLSLTLTLGEADHNHYGYPHSSS",
# PLPP4_HUMAN
"MRELAIEIGVRALLFGVFVFTEFLDPFQRVIQPEEIWLYKNPLVQSDNIPTRLMFAISFLTPLAVICVVKIIRRTDKTEIKEAFLAVSLALALNGVCTNTIKLIVGRPRPDFFYRCFPDGVMNSEMHCTGDPDLVSEGRKSFPSIHSSFAFSGLGFTTFYLAGKLHCFTESGRGKSWRLCAAILPLYCAMMIALSRMCDYKHHWQDSFVGGVIGLIFAYICYRQHYPPLANTACHKPYVSLRVPASLKKEERPTADSAPSLPLEGITEGPV",
# PLPP5_HUMAN
"MGKAAAAVAFGAEVGVRLALFAAFLVTELLPPFQRLIQPEEMWLYRNPYVEAEYFPTKPMFVIAFLSPLSLIFLAKFLKKADTRDSRQACLAASLALALNGVFTNTIKLIVGRPRPDFFYRCFPDGLAHSDLMCTGDKDVVNEGRKSFPSGHSSFAFAGLAFASFYLAGKLHCFTPQGRGKSWRFCAFLSPLLFAAVIALSRTCDYKHHWQDVLVGSMIGMTFAYVCYRQYYPPLTDAECHKPFQDKLVLSTAQKPGDSYCFDI",
# PLPP7_HUMAN
"MPASQSRARARDRNNVLNRAEFLSLNQPPKGGPEPRSSGRKASGPSAQPPPAGDGARERRQSQQLPEEDCMQLNPSFKGIAFNSLLAIDICMSKRLGVCAGRAASWASARSMVKLIGITGHGIPWIGGTILCLVKSSTLAGQEVLMNLLLALLLDIMTVAGVQKLIKRRGPYETSPSLLDYLTMDIYAFPAGHASRAAMVSKFFLSHLVLAVPLRVLLVLWALCVGLSRVMIGRHHVTDVLSGFVIGYLQFRLVELVWMPSSTCQMLISAW",
# SGPP1_HUMAN
"MSLRQRLAQLVGRLQDPQKVARFQRLCGVEAPPRRSADRREDEKAEAPLAGDPRLRGRQPGAPGGPQPPGSDRNQCPAKPDGGGAPNGVRNGLAAELGPASPRRAGALRRNSLTGEEGQLARVSNWPLYCLFCFGTELGNELFYILFFPFWIWNLDPLVGRRLVVIWVLVMYLGQCTKDIIRWPRPASPPVVKLEVFYNSEYSMPSTHAMSGTAIPISMVLLTYGRWQYPLIYGLILIPCWCSLVCLSRIYMGMHSILDIIAGFLYTILILAVFYPFVDLIDNFNQTHKYAPFIIIGLHLALGIFSFTLDTWSTSRGDTAEILGSGAGIACGSHVTYNMGLVLDPSLDTLPLAGPPITVTLFGKAILRILIGMVFVLIIRDVMKKITIPLACKIFNIPCDDIRKARQHMEVELPYRYITYGMVGFSITFFVPYIFFFIGIS",
# SGPP2_HUMAN
"MAELLRSLQDSQLVARFQRRCGLFPAPDEGPRENGADPTERAARVPGVEHLPAANGKGGEAPANGLRRAAAPEAYVQKYVVKNYFYYYLFQFSAALGQEVFYITFLPFTHWNIDPYLSRRLIIIWVLVMYIGQVAKDVLKWPRPSSPPVVKLEKRLIAEYGMPSTHAMAATAIAFTLLISTMDRYQYPFVLGLVMAVVFSTLVCLSRLYTGMHTVLDVLGGVLITALLIVLTYPAWTFIDCLDSASPLFPVCVIVVPFFLCYNYPVSDYYSPTRADTTTILAAGAGVTIGFWINHFFQLVSKPAESLPVIQNIPPLTTYMLVLGLTKFAVGIVLILLVRQLVQNLSLQVLYSWFKVVTRNKEARRRLEIEVPYKFVTYTSVGICATTFVPMLHRFLGLP",
# BPNT1_HUMAN
"MASSNTVLMRLVASAYSIAQKAGMIVRRVIAEGDLGIVEKTCATDLQTKADRLAQMSICSSLARKFPKLTIIGEEDLPSEEVDQELIEDSQWEEILKQPCPSQYSAIKEEDLVVWVDPLDGTKEYTEGLLDNVTVLIGIAYEGKAIAGVINQPYYNYEAGPDAVLGRTIWGVLGLGAFGFQLKEVPAGKHIITTTRSHSNKLVTDCVAAMNPDAVLRVGGAGNKIIQLIEGKASAYVFASPGCKKWDTCAPEVILHAVGGKLTDIHGNVLQYHKDVKHMNSAGVLATLRNYDYYASRVPESIKNALVP",
# F16P1_HUMAN
"MADQAPFDTDVNTLTRFVMEEGRKARGTGELTQLLNSLCTAVKAISSAVRKAGIAHLYGIAGSTNVTGDQVKKLDVLSNDLVMNMLKSSFATCVLVSEEDKHAIIVEPEKRGKYVVCFDPLDGSSNIDCLVSVGTIFGIYRKKSTDEPSEKDALQPGRNLVAAGYALYGSATMLVLAMDCGVNCFMLDPAIGEFILVDKDVKIKKKGKIYSLNEGYARDFDPAVTEYIQRKKFPPDNSAPYGARYVGSMVADVHRTLVYGGIFLYPANKKSPNGKLRLLYECNPMAYVMEKAGGMATTGKEAVLDVIPTDIHQRAPVILGSPDDVLEFLKVYEKHSAQ",
# F16P2_HUMAN
"MTDRSPFETDMLTLTRYVMEKGRQAKGTGELTQLLNSMLTAIKAISSAVRKAGLAHLYGIAGSVNVTGDEVKKLDVLSNSLVINMVQSSYSTCVLVSEENKDAIITAKEKRGKYVVCFDPLDGSSNIDCLASIGTIFAIYRKTSEDEPSEKDALQCGRNIVAAGYALYGSATLVALSTGQGVDLFMLDPALGEFVLVEKDVKIKKKGKIYSLNEGYAKYFDAATTEYVQKKKFPEDGSAPYGARYVGSMVADVHRTLVYGGIFLYPANQKSPKGKLRLLYECNPVAYIIEQAGGLATTGTQPVLDVKPEAIHQRVPLILGSPEDVQEYLTCVQKNQAGS",
# PCX4_HUMAN
"MSPDVPLLNDYKQDFFLKRFPQTVLGGPRFKLGYCAPPYIYVNQIILFLMPWVWGGVGTLLYQLGILKDYYTAALSGGLMLFTAFVIQFTSLYAKNKSTTVERILTTDILAEEDEHEFTSCTGAETVKFLIPGKKYVANTVFHSILAGLACGLGTWYLLPNRITLLYGSTGGTALLFFFGWMTLCIAEYSLIVNTATETATFQTQDTYEIIPLMRPLYIFFFVSVDLAHRFVVNMPALEHMNQILHILFVFLPFLWALGTLPPPDALLLWAMEQVLEFGLGGSSMSTHLRLLVMFIMSAGTAIASYFIPSTVGVVLFMTGFGFLLSLNLSDMGHKIGTKSKDLPSGPEKHFSWKECLFYIIILVLALLETSLLHHFAGFSQISKSNSQAIVGYGLMILLIILWILREIQSVYIIGIFRNPFYPKDVQTVTVFFEKQTRLMKIGIVRRILLTLVSPFAMIAFLSLDSSLQGLHSVSVCIGFTRAFRMVWQNTENALLETVIVSTVHLISSTDIWWNRSLDTGLRLLLVGIIRDRLIQFISKLQFAVTVLLTSWTEKKQRRKTTATLCILNIVFSPFVLVIIVFSTLLSSPLLPLFTLPVFLVGFPRPIQSWPGAAGTTACVCADTVYYYQMVPRLTAVLQTAMAAGSLGLLLPGSHYLGRFQDRLMWIMILECGYTYCSINIKGLELQETSCHTAEARRVDEVFEDAFEQEYTRVCSLNEHFGNVLTPCTVLPVKLYSDARNVLSGIIDSHENLKEFKGDLIKVLVWILVQYCSKRPGMKENVHNTENKGKAPLMLPALNTLPPPKSPEDIDSLNSETFNDWSDDNIFDDEPTIKKVIEEKHQLKDLPGTNLFIPGSVESQRVGDHSTGTVPENDLYKAVLLGYPAVDKGKQEDMPYIPLMEFSCSHSHLVCLPAEWRTSCMPSSKMKEMSSLFPEDWYQFVLRQLECYHSEEKASNVLEEIAKDKVLKDFYVHTVMTCYFSLFGIDNMAPSPGHILRVYGGVLPWSVALDWLTEKPELFQLALKAFRYTLKLMIDKASLGPIEDFRELIKYLEEYERDWYIGLVSDEKWKEAILQEKPYLFSLGYDSNMGIYTGRVLSLQELLIQVGKLNPEAVRGQWANLSWELLYATNDDEERYSIQAHPLLLRNLTVQAAEPPLGYPIYSSKPLHIHLY",
# IMPA1_HUMAN
"MADPWQECMDYAVTLARQAGEVVCEAIKNEMNVMLKSSPVDLVTATDQKVEKMLISSIKEKYPSHSFIGEESVAAGEKSILTDNPTWIIDPIDGTTNFVHRFPFVAVSIGFAVNKKIEFGVVYSCVEGKMYTARKGKGAFCNGQKLQVSQQEDITKSLLVTELGSSRTPETVRMVLSNMEKLFCIPVHGIRSVGTAAVNMCLVATGGADAYYEMGIHCWDVAGAGIIVTEAGGVLMDVTGGPFDLMSRRVIAANNRILAERIAKEIQVIPLQRDDED",
# IMPA2_HUMAN
"MKPSGEDQAALAAGPWEECFQAAVQLALRAGQIIRKALTEEKRVSTKTSAADLVTETDHLVEDLIISELRERFPSHRFIAEEAAASGAKCVLTHSPTWIIDPIDGTCNFVHRFPTVAVSIGFAVRQELEFGVIYHCTEERLYTGRRGRGAFCNGQRLRVSGETDLSKALVLTEIGPKRDPATLKLFLSNMERLLHAKAHGVRVIGSSTLALCHLASGAADAYYQFGLHCWDLAAATVIIREAGGIVIDTSGGPLDLMACRVVAASTREMAMLIAQALQTINYGRDDEK",
# IMPA3_HUMAN
"MAPMGIRLSPLGVAVFCLLGLGVLYHLYSGFLAGRFSLFGLGGEPGGGAAGPAAAADGGTVDLREMLAVSVLAAVRGGDEVRRVRESNVLHEKSKGKTREGAEDKMTSGDVLSNRKMFYLLKTAFPSVQINTEEHVDAADQEVILWDHKIPEDILKEVTTPKEVPAESVTVWIDPLDATQEYTEDLRKYVTTMVCVAVNGKPMLGVIHKPFSEYTAWAMVDGGSNVKARSSYNEKTPRIVVSRSHSGMVKQVALQTFGNQTTIIPAGGAGYKVLALLDVPDKSQEKADLYIHVTYIKKWDICAGNAILKALGGHMTTLSGEEISYTGSDGIEGGLLASIRMNHQALVRKLPDLEKTGHK",
# INPP_HUMAN
"MSDILRELLCVSEKAANIARACRQQEALFQLLIEEKKEGEKNKKFAVDFKTLADVLVQEVIKQNMENKFPGLEKNIFGEESNEFTNDWGEKITLRLCSTEEETAELLSKVLNGNKVASEALARVVHQDVAFTDPTLDSTEINVPQDILGIWVDPIDSTYQYIKGSADIKSNQGIFPCGLQCVTILIGVYDIQTGVPLMGVINQPFVSRDPNTLRWKGQCYWGLSYMGTNMHSLQLTISRRNGSETHTGNTGSEAAFSPSFSAVISTSEKETIKAALSRVCGDRIFGAAGAGYKSLCVVQGLVDIYIFSEDTTFKWDSCAAHAILRAMGGGIVDLKECLERNPETGLDLPQLVYHVENEGAAGVDRWANKGGLIAYRSRKRLETFLSLLVQNLAPAETHT",
# I5P1_HUMAN
"MAGKAAAPGTAVLLVTANVGSLFDDPENLQKNWLREFYQVVHTHKPHFMALHCQEFGGKNYEASMSHVDKFVKELLSSDAMKEYNRARVYLDENYKSQEHFTALGSFYFLHESLKNIYQFDFKAKKYRKVAGKEIYSDTLESTPMLEKEKFPQDYFPECKWSRKGFIRTRWCIADCAFDLVNIHLFHDASNLVAWETSPSVYSGIRHKALGYVLDRIIDQRFEKVSYFVFGDFNFRLDSKSVVETLCTKATMQTVRAADTNEVVKLIFRESDNDRKVMLQLEKKLFDYFNQEVFRDNNGTALLEFDKELSVFKDRLYELDISFPPSYPYSEDARQGEQYMNTRCPAWCDRILMSPSAKELVLRSESEEKVVTYDHIGPNVCMGDHKPVFLAFRIMPGAGKPHAHVHKCCVVQ",
# I5P2_HUMAN
"MDQSVAIQETLAEGEYCVIAVQGVLCEGDSRQSRLLGLVRYRLEHGGQEHALFLYTHRRMAITGDDVSLDQIVPVSRDFTLEEVSPDGELYILGSDVTVQLDTAELSLVFQLPFGSQTRMFLHEVARACPGFDSATRDPEFLWLSRYRCAELELEMPTPRGCNSALVTWPGYATIGGGRYPSRKKRWGLEEARPQGAGSVLFWGGAMEKTGFRLMERAHGGGFVWGRSARDGRRDEELEEAGREMSAAAGSRERNTAGGSNFDGLRPNGKGVPMDQSSRGQDKPESLQPRQNKSKSEITDMVRSSTITVSDKAHILSMQKFGLRDTIVKSHLLQKEEDYTYIQNFRFFAGTYNVNGQSPKECLRLWLSNGIQAPDVYCVGFQELDLSKEAFFFHDTPKEEEWFKAVSEGLHPDAKYAKVKLIRLVGIMLLLYVKQEHAAYISEVEAETVGTGIMGRMGNKGGVAIRFQFHNTSICVVNSHLAAHIEEYERRNQDYKDICSRMQFCQPDPSLPPLTISNHDVILWLGDLNYRIEELDVEKVKKLIEEKDFQMLYAYDQLKIQVAAKTVFEGFTEGELTFQPTYKYDTGSDDWDTSEKCRAPAWCDRILWKGKNITQLSYQSHMALKTSDHKPVSSVFDIGVRVVNDELYRKTLEEIVRSLDKMENANIPSVSLSKREFCFQNVKYMQLKVESFTIHNGQVPCHFEFINKPDEESYCKQWLNANPSRGFLLPDSDVEIDLELFVNKMTATKLNSGEDKIEDILVLHLDRGKDYFLSVSGNYLPSCFGSPIHTLCYMREPILDLPLETISELTLMPVWTGDDGSQLDSPMEIPKELWMMVDYLYRNAVQQEDLFQQPGLRSEFEHIRDCLDTGMIDNLSASNHSVAEALLLFLESLPEPVICYSTYHNCLECSGNYTASKQVISTLPIFHKNVFHYLMAFLRELLKNSAKNHLDENILASIFGSLLLRNPAGHQKLDMTEKKKAQEFIHQFLCNPL",
# SHIP1_HUMAN
"MVPCWNHGNITRSKAEELLSRTGKDGSFLVRASESISRAYALCVLYRNCVYTYRILPNEDDKFTVQASEGVSMRFFTKLDQLIEFYKKENMGLVTHLQYPVPLEEEDTGDDPEEDTVESVVSPPELPPRNIPLTASSCEAKEVPFSNENPRATETSRPSLSETLFQRLQSMDTSGLPEEHLKAIQDYLSTQLAQDSEFVKTGSSSLPHLKKLTTLLCKELYGEVIRTLPSLESLQRLFDQQLSPGLRPRPQVPGEANPINMVSKLSQLTSLLSSIEDKVKALLHEGPESPHRPSLIPPVTFEVKAESLGIPQKMQLKVDVESGKLIIKKSKDGSEDKFYSHKKILQLIKSQKFLNKLVILVETEKEKILRKEYVFADSKKREGFCQLLQQMKNKHSEQPEPDMITIFIGTWNMGNAPPPKKITSWFLSKGQGKTRDDSADYIPHDIYVIGTQEDPLSEKEWLEILKHSLQEITSVTFKTVAIHTLWNIRIVVLAKPEHENRISHICTDNVKTGIANTLGNKGAVGVSFMFNGTSLGFVNSHLTSGSEKKLRRNQNYMNILRFLALGDKKLSPFNITHRFTHLFWFGDLNYRVDLPTWEAETIIQKIKQQQYADLLSHDQLLTERREQKVFLHFEEEEITFAPTYRFERLTRDKYAYTKQKATGMKYNLPSWCDRVLWKSYPLVHVVCQSYGSTSDIMTSDHSPVFATFEAGVTSQFVSKNGPGTVDSQGQIEFLRCYATLKTKSQTKFYLEFHSSCLESFVKSQEGENEEGSEGELVVKFGETLPKLKPIISDPEYLLDQHILISIKSSDSDESYGEGCIALRLEATETQLPIYTPLTHHGELTGHFQGEIKLQTSQGKTREKLYDFVKTERDESSGPKTLKSLTSHDPMKQWEVTSRAPPCSGSSITEIINPNYMGVGPFGPPMPLHVKQTLSPDQQPTAWSYDQPPKDSPLGPCRGESPPTPPGQPPISPKKFLPSTANRGLPPRTQESRPSDLGKNAGDTLPQEDLPLTKPEMFENPLYGSLSSFPKPAPRKDQESPKMPRKEPPPCPEPGILSPSIVLTKAQEADRGEGPGKQVPAPRLRSFTCSSSAEGRAAGGDKSQGKPKTPVSSQAPVPAKRPIKPSRSEINQQTPPTPTPRPPLPVKSPAVLHLQHSKGRDYRDNTELPHHGKHRPEEGPPGPLGRTAMQ",
# MPPA_HUMAN
"MAAVVLAATRLLRGSGSWGCSRLRFGPPAYRRFSSGGAYPNIPLSSPLPGVPKPVFATVDGQEKFETKVTTLDNGLRVASQNKFGQFCTVGILINSGSRYEAKYLSGIAHFLEKLAFSSTARFDSKDEILLTLEKHGGICDCQTSRDTTMYAVSADSKGLDTVVALLADVVLQPRLTDEEVEMTRMAVQFELEDLNLRPDPEPLLTEMIHEAAYRENTVGLHRFCPTENVAKINREVLHSYLRNYYTPDRMVLAGVGVEHEHLVDCARKYLLGVQPAWGSAEAVDIDRSVAQYTGGIAKLERDMSNVSLGPTPIPELTHIMVGLESCSFLEEDFIPFAVLNMMMGGGGSFSAGGPGKGMFSRLYLNVLNRHHWMYNATSYHHSYEDTGLLCIHASADPRQVREMVEIITKEFILMGGTVDTVELERAKTQLTSMLMMNLESRPVIFEDVGRQVLATRSRKLPHELCTLIRNVKPEDVKRVASKMLRGKPAVAALGDLTDLPTYEHIQTALSSKDGRLPRTYRLFR",
# INP5E_HUMAN
"MPSKAENLRPSEPAPQPPEGRTLQGQLPGAPPAQRAGSPPDAPGSESPALACSTPATPSGEDPPARAAPIAPRPPARPRLERALSLDDKGWRRRRFRGSQEDLEARNGTSPSRGSVQSEGPGAPAHSCSPPCLSTSLQEIPKSRGVLSSERGSPSSGGNPLSGVASSSPNLPHRDAAVAGSSPRLPSLLPPRPPPALSLDIASDSLRTANKVDSDLADYKLRAQPLLVRAHSSLGPGRPRSPLACDDCSLRSAKSSFSLLAPIRSKDVRSRSYLEGSLLASGALLGADELARYFPDRNVALFVATWNMQGQKELPPSLDEFLLPAEADYAQDLYVIGVQEGCSDRREWETRLQETLGPHYVLLSSAAHGVLYMSLFIRRDLIWFCSEVECSTVTTRIVSQIKTKGALGISFTFFGTSFLFITSHFTSGDGKVAERLLDYTRTVQALVLPRNVPDTNPYRSSAADVTTRFDEVFWFGDFNFRLSGGRTVVDALLCQGLVVDVPALLQHDQLIREMRKGSIFKGFQEPDIHFLPSYKFDIGKDTYDSTSKQRTPSYTDRVLYRSRHKGDICPVSYSSCPGIKTSDHRPVYGLFRVKVRPGRDNIPLAAGKFDRELYLLGIKRRISKEIQRQQALQSQNSSTICSVS",
# PI5PA_HUMAN
"MEGQSSRGSRRPGTRAGLGSLPMPQGVAQTGAPSKVDSSFQLPAKKNAALGPSEPRLALAPVGPRAAMSASSEGPRLALASPRPILAPLCTPEGQKTATAHRSSSLAPTSVGQLVMSASAGPKPPPATTGSVLAPTSLGLVMPASAGPRSPPVTLGPNLAPTSRDQKQEPPASVGPKPTLAASGLSLALASEEQPPELPSTPSPVPSPVLSPTQEQALAPASTASGAASVGQTSARKRDAPAPRPLPASEGHLQPPAQTSGPTGSPPCIQTSPDPRLSPSFRARPEALHSSPEDPVLPRPPQTLPLDVGQGPSEPGTHSPGLLSPTFRPGAPSGQTVPPPLPKPPRSPSRSPSHSPNRSPCVPPAPDMALPRLGTQSTGPGRCLSPNLQAQEAPAPVTTSSSTSTLSSSPWSAQPTWKSDPGFRITVVTWNVGTAMPPDDVTSLLHLGGGDDSDGADMIAIGLQEVNSMLNKRLKDALFTDQWSELFMDALGPFNFVLVSSVRMQGVILLLFAKYYHLPFLRDVQTDCTRTGLGGYWGNKGGVSVRLAAFGHMLCFLNCHLPAHMDKAEQRKDNFQTILSLQQFQGPGAQGILDHDLVFWFGDLNFRIESYDLHFVKFAIDSDQLHQLWEKDQLNMAKNTWPILKGFQEGPLNFAPTFKFDVGTNKYDTSAKKRKPAWTDRILWKVKAPGGGPSPSGRKSHRLQVTQHSYRSHMEYTVSDHKPVAAQFLLQFAFRDDMPLVRLEVADEWVRPEQAVVRYRMETVFARSSWDWIGLYRVGFRHCKDYVAYVWAKHEDVDGNTYQVTFSEESLPKGHGDFILGYYSHNHSILIGITEPFQISLPSSELASSSTDSSGTSSEGEDDSTLELLAPKSRSPSPGKSKRHRSRSPGLARFPGLALRPSSRERRGASRSPSPQSRRLSRVAPDRSSNGSSRGSSEEGPSGLPGPWAFPPAVPRSLGLLPALRLETVDPGGGGSWGPDREALAPNSLSPSPQGHRGLEEGGLGP",
# INP5K_HUMAN
"MSSRKLSGPKGRRLSIHVVTWNVASAAPPLDLSDLLQLNNRNLNLDIYVIGLQELNSGIISLLSDAAFNDSWSSFLMDVLSPLSFIKVSHVRMQGILLLVFAKYQHLPYIQILSTKSTPTGLFGYWGNKGGVNICLKLYGYYVSIINCHLPPHISNNYQRLEHFDRILEMQNCEGRDIPNILDHDLIIWFGDMNFRIEDFGLHFVRESIKNRCYGGLWEKDQLSIAKKHDPLLREFQEGRLLFPPTYKFDRNSNDYDTSEKKRKPAWTDRILWRLKRQPCAGPDTPIPPASHFSLSLRGYSSHMTYGISDHKPVSGTFDLELKPLVSAPLIVLMPEDLWTVENDMMVSYSSTSDFPSSPWDWIGLYKVGLRDVNDYVSYAWVGDSKVSCSDNLNQVYIDISNIPTTEDEFLLCYYSNSLRSVVGISRPFQIPPGSLREDPLGEAQPQI",
# SHIP2_HUMAN
"MASACGAPGPGGALGSQAPSWYHRDLSRAAAEELLARAGRDGSFLVRDSESVAGAFALCVLYQKHVHTYRILPDGEDFLAVQTSQGVPVRRFQTLGELIGLYAQPNQGLVCALLLPVEGEREPDPPDDRDASDGEDEKPPLPPRSGSTSISAPTGPSSPLPAPETPTAPAAESAPNGLSTVSHDYLKGSYGLDLEAVRGGASHLPHLTRTLATSCRRLHSEVDKVLSGLEILSKVFDQQSSPMVTRLLQQQNLPQTGEQELESLVLKLSVLKDFLSGIQKKALKALQDMSSTAPPAPQPSTRKAKTIPVQAFEVKLDVTLGDLTKIGKSQKFTLSVDVEGGRLVLLRRQRDSQEDWTTFTHDRIRQLIKSQRVQNKLGVVFEKEKDRTQRKDFIFVSARKREAFCQLLQLMKNKHSKQDEPDMISVFIGTWNMGSVPPPKNVTSWFTSKGLGKTLDEVTVTIPHDIYVFGTQENSVGDREWLDLLRGGLKELTDLDYRPIAMQSLWNIKVAVLVKPEHENRISHVSTSSVKTGIANTLGNKGAVGVSFMFNGTSFGFVNCHLTSGNEKTARRNQNYLDILRLLSLGDRQLNAFDISLRFTHLFWFGDLNYRLDMDIQEILNYISRKEFEPLLRVDQLNLEREKHKVFLRFSEEEISFPPTYRYERGSRDTYAWHKQKPTGVRTNVPSWCDRILWKSYPETHIICNSYGCTDDIVTSDHSPVFGTFEVGVTSQFISKKGLSKTSDQAYIEFESIEAIVKTASRTKFFIEFYSTCLEEYKKSFENDAQSSDNINFLKVQWSSRQLPTLKPILADIEYLQDQHLLLTVKSMDGYESYGECVVALKSMIGSTAQQFLTFLSHRGEETGNIRGSMKVRVPTERLGTRERLYEWISIDKDEAGAKSKAPSVSRGSQEPRSGSRKPAFTEASCPLSRLFEEPEKPPPTGRPPAPPRAAPREEPLTPRLKPEGAPEPEGVAAPPPKNSFNNPAYYVLEGVPHQLLPPEPPSPARAPVPSATKNKVAITVPAPQLGHHRHPRVGEGSSSDEESGGTLPPPDFPPPPLPDSAIFLPPSLDPLPGPVVRGRGGAEARGPPPPKAHPRPPLPPGPSPASTFLGEVASGDDRSCSVLQMAKTLSEVDYAPAGPARSALLPGPLELQPPRGLPSDYGRPLSFPPPRIRESIQEDLAEEAPCLQGGRASGLGEAGMSAWLRAIGLERYEEGLVHNGWDDLEFLSDITEEDLEEAGVQDPAHKRLLLDTLQLSK",
# OCRL_HUMAN
"MEPPLPVGAQPLATVEGMEMKGPLREPCALTLAQRNGQYELIIQLHEKEQHVQDIIPINSHFRCVQEAEETLLIDIASNSGCKIRVQGDWIRERRFEIPDEEHCLKFLSAVLAAQKAQSQLLVPEQKDSSSWYQKLDTKDKPSVFSGLLGFEDNFSSMNLDKKINSQNQPTGIHREPPPPPFSVNKMLPREKEASNKEQPKVTNTMRKLFVPNTQSGQREGLIKHILAKREKEYVNIQTFRFFVGTWNVNGQSPDSGLEPWLNCDPNPPDIYCIGFQELDLSTEAFFYFESVKEQEWSMAVERGLHSKAKYKKVQLVRLVGMMLLIFARKDQCRYIRDIATETVGTGIMGKMGNKGGVAVRFVFHNTTFCIVNSHLAAHVEDFERRNQDYKDICARMSFVVPNQTLPQLNIMKHEVVIWLGDLNYRLCMPDANEVKSLINKKDLQRLLKFDQLNIQRTQKKAFVDFNEGEIKFIPTYKYDSKTDRWDSSGKCRVPAWCDRILWRGTNVNQLNYRSHMELKTSDHKPVSALFHIGVKVVDERRYRKVFEDSVRIMDRMENDFLPSLELSRREFVFENVKFRQLQKEKFQISNNGQVPCHFSFIPKLNDSQYCKPWLRAEPFEGYLEPNETVDISLDVYVSKDSVTILNSGEDKIEDILVLHLDRGKDYFLTISGNYLPSCFGTSLEALCRMKRPIREVPVTKLIDLEEDSFLEKEKSLLQMVPLDEGASERPLQVPKEIWLLVDHLFKYACHQEDLFQTPGMQEELQQIIDCLDTSIPETIPGSNHSVAEALLIFLEALPEPVICYELYQRCLDSAYDPRICRQVISQLPRCHRNVFRYLMAFLRELLKFSEYNSVNANMIATLFTSLLLRPPPNLMARQTPSDRQRAIQFLLGFLLGSEED",
# 5NT1A_HUMAN
"MEPGQPREPQEPREPGPGAETAAAPVWEEAKIFYDNLAPKKKPKSPKPQNAVTIAVSSRALFRMDEEQQIYTEQGVEEYVRYQLEHENEPFSPGPAFPFVKALEAVNRRLRELYPDSEDVFDIVLMTNNHAQVGVRLINSINHYDLFIERFCMTGGNSPICYLKAYHTNLYLSADAEKVREAIDEGIAAATIFSPSRDVVVSQSQLRVAFDGDAVLFSDESERIVKAHGLDRFFEHEKAHENKPLAQGPLKGFLEALGRLQKKFYSKGLRLECPIRTYLVTARSAASSGARALKTLRSWGLETDEALFLAGAPKGPLLEKIRPHIFFDDQMFHVAGAQEMGTVAAHVPYGVAQTPRRTAPAKQAPSAQ",
# 5NT1B_HUMAN
"MSQTSLKQKKNEPGMRSSKESLEAEKRKESDKTGVRLSNQMRRAVNPNHSLRCCPFQGHSSCRRCLCAAEGTALGPCHTIRIYIHMCLLWEQGQQITMMRGSQESSLRKTDSRGYLVRSQWSRISRSPSTKAPSIDEPRSRNTSAKLPSSSTSSRTPSTSPSLHDSSPPPLSGQPSLQPPASPQLPRSLDSRPPTPPEPDPGSRRSTKMQENPEAWAQGIVREIRQTRDSQPLEYSRTSPTEWKSSSQRRGIYPASTQLDRNSLSEQQQQQREDEDDYEAAYWASMRSFYEKNPSCSRPWPPKPKNAITIALSSCALFNMVDGRKIYEQEGLEKYMEYQLTNENVILTPGPAFRFVKALQYVNARLRDLYPDEQDLFDIVLMTNNHAQVGVRLINSVNHYGLLIDRFCLTGGKDPIGYLKAYLTNLYIAADSEKVQEAIQEGIASATMFDGAKDMAYCDTQLRVAFDGDAVLFSDESEHFTKEHGLDKFFQYDTLCESKPLAQGPLKGFLEDLGRLQKKFYAKNERLLCPIRTYLVTARSAASSGARVLKTLRRWGLEIDEALFLAGAPKSPILVKIRPHIFFDDHMFHIEGAQRLGSIAAYGFNKKFSS",
# 5NTC_HUMAN
"MSTSWSDRLQNAADMPANMDKHALKKYRREAYHRVFVNRSLAMEKIKCFGFDMDYTLAVYKSPEYESLGFELTVERLVSIGYPQELLSFAYDSTFPTRGLVFDTLYGNLLKVDAYGNLLVCAHGFNFIRGPETREQYPNKFIQRDDTERFYILNTLFNLPETYLLACLVDFFTNCPRYTSCETGFKDGDLFMSYRSMFQDVRDAVDWVHYKGSLKEKTVENLEKYVVKDGKLPLLLSRMKEVGKVFLATNSDYKYTDKIMTYLFDFPHGPKPGSSHRPWQSYFDLILVDARKPLFFGEGTVLRQVDTKTGKLKIGTYTGPLQHGIVYSGGSSDTICDLLGAKGKDILYIGDHIFGDILKSKKRQGWRTFLVIPELAQELHVWTDKSSLFEELQSLDIFLAELYKHLDSSSNERPDISSIQRRIKKVTHDMDMCYGMMGSLFRSGSRQTLFASQVMRYADLYAASFINLLYYPFSYLFRAAHVLMPHESTVEHTHVDINEMESPLATRNRTSVDFKDTDYKRHQLTRSISEIKPPNLFPLAPQEITHCHDEDDDEEEEEEEE",
# NT5D1_HUMAN
"MAQHFSLAACDVVGFDLDHTLCRYNLPESAPLIYNSFAQFLVKEKGYDKELLNVTPEDWDFCCKGLALDLEDGNFLKLANNGTVLRASHGTKMMTPEVLAEAYGKKEWKHFLSDTGMACRSGKYYFYDNYFDLPGALLCARVVDYLTKLNNGQKTFDFWKDIVAAIQHNYKMSAFKENCGIYFPEIKRDPGRYLHSCPESVKKWLRQLKNAGKILLLITSSHSDYCRLLCEYILGNDFTDLFDIVITNALKPGFFSHLPSQRPFRTLENDEEQEALPSLDKPGWYSQGNAVHLYELLKKMTGKPEPKVVYFGDSMHSDIFPARHYSNWETVLILEELRGDEGTRSQRPEESEPLEKKGKYEGPKAKPLNTSSKKWGSFFIDSVLGLENTEDSLVYTWSCKRISTYSTIAIPSIEAIAELPLDYKFTRFSSSNSKTAGYYPNPPLVLSSDETLISK",
# NT5D2_HUMAN
"MRVESGSAQERGILLESLSTLLEKTTASHEGRAPGNRELTDLLPPEVCSLLNPAAIYANNEISLRDVEVYGFDYDYTLAQYADALHPEIFSTARDILIEHYKYPEGIRKYDYNPSFAIRGLHYDIQKSLLMKIDAFHYVQLGTAYRGLQPVPDEEVIELYGGTQHIPLYQMSGFYGKGPSIKQFMDIFSLPEMALLSCVVDYFLGHSLEFDQAHLYKDVTDAIRDVHVKGLMYQWIEQDMEKYILRGDETFAVLSRLVAHGKQLFLITNSPFSFVDKGMRHMVGPDWRQLFDVVIVQADKPSFFTDRRKPFRKLDEKGSLQWDRITRLEKGKIYRQGNLFDFLRLTEWRGPRVLYFGDHLYSDLADLMLRHGWRTGAIIPELEREIRIINTEQYMHSLTWQQALTGLLERMQTYQDAESRQVLAAWMKERQELRCITKALFNAQFGSIFRTFHNPTYFSRRLVRFSDLYMASLSCLLNYRVDFTFYPRRTPLQHEAPLWMDQLCTGCMKTPFLGDMAHIR",
# NT5D3_HUMAN
"MTMAAAAVVARGAGARAATAAALRGGCGTAARGRPCAGPARPLCTAPGTAPDMKRYLWERYREAKRSTEELVPSIMSNLLNPDAIFSNNEMSLSDIEIYGFDYDYTLVFYSKHLHTLIFNAARDLLINEHRYPAEIRKYEYDPNFAIRGLHYDVQRAVLMKIDAFHYIQLGTVYRGLSVVPDEEVIEMYEGSHVPLEQMSDFYGKSSHGNTMKQFMDIFSLPEMTLLSCVNEYFLKNNIDYEPVHLYKDVKDSIRDVHIKGIMYRAIEADIEKYICYAEQTRAVLAKLADHGKKMFLITNSPSSFVDKGMSYIVGKDWRDLFDVVIVQAEKPNFFNDKRRPFRKMNEKGVLLWDKIHKLQKGQIYKQGNLYEFLKLTGWRGSRVLYFGDHIYSDLADLTLKHGWRTGAIIPELRSELKIMNTEQYIQTMTWLQTLTGLLEQMQVHRDAESQLVLQEWKKERKEMREMTKSFFNAQFGSLFRTDQNPTYFLRRLSRFADIYMASLSCLLNYDVSHTFYPRRTPLQHELPAWSERPPTFGTPLLQEAQAK",
# NT5D4_HUMAN
"MPAWIFVNRSLALGKIRCFGFDMDYTLAAYKSPAYEALTFELLLERLVCIGYPHEILRYTYDPTFPTRRLVFDELYGNLLKVDAHGNVLLGAYGFTFLSEAEIWSFYPSKFIQRDDLQCFYILNMLFNLPETYLYACLVDFFSGCSRYTNCDTGYQHGNLFMSFRSLFQDVTDAMNNIHQSGCLKKTLEDLEKYVKKDPRLPILLGKMKEVGKVFLATNSSYNYTNAIMTYLFSISEAEASGRPWRSYFDLIVVDTQKPHFFAEGLVLRQVNTDSGKLHVGTYTGPHQHCAVYSGGSSDMVCELLGVRGMDILYIGDHIFGDILKSKKRQGWRTCLVVPELSWELDIWAQEKERLEELKRLDTHLADIYQHMDGSSCELQVINFTKREIQMPHESVVEQEQANLDPASCLLSCNQRSLPAKSCLSSAI",
# PMM1_HUMAN
"MAVTAQAARRKERVLCLFDVDGTLTPARQKIDPEVAAFLQKLRSRVQIGVVGGSDYCKIAEQLGDGDEVIEKFDYVFAENGTVQYKHGRLLSKQTIQNHLGEELLQDLINFCLSYMALLRLPKKRGTFIEFRNGMLNISPIGRSCTLEERIEFSELDKKEKIREKFVEALKTEFAGKGLRFSRGGMISFDVFPEGWDKRYCLDSLDQDSFDTIHFFGNETSPGGNDFEIFADPRTVGHSVVSPQDTVQRCREIFFPETAHEA",
# PMM2_HUMAN
"MAAPGPALCLFDVDGTLTAPRQKITKEMDDFLQKLRQKIKIGVVGGSDFEKVQEQLGNDVVEKYDYVFPENGLVAYKDGKLLCRQNIQSHLGEALIQDLINYCLSYIAKIKLPKKRGTFIEFRNGMLNVSPIGRSCSQEERIEFYELDKKENIRQKFVADLRKEFAGKGLTFSIGGQISFDVFPDGWDKRYCLRHVENDGYKTIYFFGDKTMPGGNDHEIFTDPRTMGYSVTAPEDTRRICELLFS",
# NT5C_HUMAN
"MARSVRVLVDMDGVLADFEAGLLRGFRRRFPEEPHVPLEQRRGFLAREQYRALRPDLADKVASVYEAPGFFLDLEPIPGALDAVREMNDLPDTQVFICTSPLLKYHHCVGEKYRWVEQHLGPQFVERIILTRDKTVVLGDLLIDDKDTVRGQEETPSWEHILFTCCHNRHLVLPPTRRRLLSWSDNWREILDSKRGAAQRE",
# NT5M_HUMAN
"MIRLGGWCARRLCSAAVPAGRRGAAGGLGLAGGRALRVLVDMDGVLADFEGGFLRKFRARFPDQPFIALEDRRGFWVSEQYGRLRPGLSEKAISIWESKNFFFELEPLPGAVEAVKEMASLQNTDVFICTSPIKMFKYCPYEKYAWVEKYFGPDFLEQIVLTRDKTVVSADLLIDDRPDITGAEPTPSWEHVLFTACHNQHLQLQPPRRRLHSWADDWKAILDSKRPC",
# ENOPH_HUMAN
"MVVLSVPAEVTVILLDIEGTTTPIAFVKDILFPYIEENVKEYLQTHWEEEECQQDVSLLRKQAEEDAHLDGAVPIPAASGNGVDDLQQMIQAVVDNVCWQMSLDRKTTALKQLQGHMWRAAFTAGRMKAEFFADVVPAVRKWREAGMKVYIYSSGSVEAQKLLFGHSTEGDILELVDGHFDTKIGHKVESESYRKIADSIGCSTNNILFLTDVTREASAAEEADVHVAVVVRPGNAGLTDDEKTYYSLITSFSELYLPSST",
# HDHD3_HUMAN
"MAHRLQIRLLTWDVKDTLLRLRHPLGEAYATKARAHGLEVEPSALEQGFRQAYRAQSHSFPNYGLSHGLTSRQWWLDVVLQTFHLAGVQDAQAVAPIAEQLYKDFSHPCTWQVLDGAEDTLRECRTRGLRLAVISNFDRRLEGILGGLGLREHFDFVLTSEAAGWPKPDPRIFQEALRLAHMEPVVAAHVGDNYLCDYQGPRAVGMHSFLVVGPQALDPVVRDSVPKEHILPSLAHLLPALDCLEGSTPGL",
# NANP_HUMAN
"MGLSRVRAVFFDLDNTLIDTAGASRRGMLEVIKLLQSKYHYKEEAEIICDKVQVKLSKECFHPYNTCITDLRTSHWEEAIQETKGGAANRKLAEECYFLWKSTRLQHMTLAEDVKAMLTELRKEVRLLLLTNGDRQTQREKIEACACQSYFDAVVVGGEQREEKPAPSIFYYCCNLLGVQPGDCVMVGDTLETDIQGGLNAGLKATVWINKNGIVPLKSSPVPHYMVSSVLELPALLQSIDCKVSMST",
# HDHD1_HUMAN
"MAAPPQPVTHLIFDMDGLLLDTERLYSVVFQEICNRYDKKYSWDVKSLVMGKKALEAAQIIIDVLQLPMSKEELVEESQTKLKEVFPTAALMPGAEKLIIHLRKHGIPFALATSSGSASFDMKTSRHKEFFSLFSHIVLGDDPEVQHGKPDPDIFLACAKRFSPPPAMEKCLVFEDAPNGVEAALAAGMQVVMVPDGNLSRDLTTKATLVLNSLQDFQPELFGLPSYE",
# LPIN1_HUMAN
"MNYVGQLAGQVFVTVKELYKGLNPATLSGCIDIIVIRQPNGNLQCSPFHVRFGKMGVLRSREKVVDIEINGESVDLHMKLGDNGEAFFVQETDNDQEVIPMHLATSPILSEGASRMECQLKRGSVDRMRGLDPSTPAQVIAPSETPSSSSVVKKRRKRRRKSQLDSLKRDDNMNTSEDEDMFPIEMSSDEAMELLESSRTLPNDIPPFQDDIPEENLSLAVIYPQSASYPNSDREWSPTPSPSGSRPSTPKSDSELVSKSTERTGQKNPEMLWLWGELPQAAKSSSPHKMKESSPLSSRKICDKSHFQAIHSESSDTFSDQSPTLVGGALLDQNKPQTEMQFVNEEDLETLGAAAPLLPMIEELKPPSASVVQTANKTDSPSRKRDKRSRHLGADGVYLDDLTDMDPEVAALYFPKNGDPSGLAKHASDNGARSANQSPQSVGSSGVDSGVESTSDGLRDLPSIAISLCGGLSDHREITKDAFLEQAVSYQQFVDNPAIIDDPNLVVKIGSKYYNWTTAAPLLLAMQAFQKPLPKATVESIMRDKMPKKGGRWWFSWRGRNTTIKEESKPEQCLAGKAHSTGEQPPQLSLATRVKHESSSSDEERAAAKPSNAGHLPLLPNVSYKKTLRLTSEQLKSLKLKNGPNDVVFSVTTQYQGTCRCEGTIYLWNWDDKVIISDIDGTITRSDTLGHILPTLGKDWTHQGIAKLYHKVSQNGYKFLYCSARAIGMADMTRGYLHWVNERGTVLPQGPLLLSPSSLFSALHREVIEKKPEKFKVQCLTDIKNLFFPNTEPFYAAFGNRPADVYSYKQVGVSLNRIFTVNPKGELVQEHAKTNISSYVRLCEVVDHVFPLLKRSHSSDFPCSDTFSNFTFWREPLPPFENQDIHSASA",
# LPIN2_HUMAN
"MNYVGQLAGQVIVTVKELYKGINQATLSGCIDVIVVQQQDGSYQCSPFHVRFGKLGVLRSKEKVIDIEINGSAVDLHMKLGDNGEAFFVEETEEEYEKLPAYLATSPIPTEDQFFKDIDTPLVKSGGDETPSQSSDISHVLETETIFTPSSVKKKKRRRKKYKQDSKKEEQAASAAAEDTCDVGVSSDDDKGAQAARGSSNASLKEEECKEPLLFHSGDHYPLSDGDWSPLETTYPQTACPKSDSELEVKPAESLLRSESHMEWTWGGFPESTKVSKRERSDHHPRTATITPSENTHFRVIPSEDNLISEVEKDASMEDTVCTIVKPKPRALGTQMSDPTSVAELLEPPLESTQISSMLDADHLPNAALAEAPSESKPAAKVDSPSKKKGVHKRSQHQGPDDIYLDDLKGLEPEVAALYFPKSESEPGSRQWPESDTLSGSQSPQSVGSAAADSGTECLSDSAMDLPDVTLSLCGGLSENGEISKEKFMEHIITYHEFAENPGLIDNPNLVIRIYNRYYNWALAAPMILSLQVFQKSLPKATVESWVKDKMPKKSGRWWFWRKRESMTKQLPESKEGKSEAPPASDLPSSSKEPAGARPAENDSSSDEGSQELEESITVDPIPTEPLSHGSTTSYKKSLRLSSDQIAKLKLHDGPNDVVFSITTQYQGTCRCAGTIYLWNWNDKIIISDIDGTITKSDALGQILPQLGKDWTHQGIAKLYHSINENGYKFLYCSARAIGMADMTRGYLHWVNDKGTILPRGPLMLSPSSLFSAFHREVIEKKPEKFKIECLNDIKNLFAPSKQPFYAAFGNRPNDVYAYTQVGVPDCRIFTVNPKGELIQERTKGNKSSYHRLSELVEHVFPLLSKEQNSAFPCPEFSSFCYWRDPIPEVDLDDLS",
# LPIN3_HUMAN
"MNYVGQLAETVFGTVKELYRGLNPATLSGGIDVLVVKQVDGSFRCSPFHVRFGKLGVLRSREKVVDIELNGEPVDLHMKLGDSGEAFFVQELESDDEHVPPGLCTSPIPWGGLSGFPSDSQLGTASEPEGLVMAGTASTGRRKRRRRRKPKQKEDAVATDSSPEELEAGAESELSLPEKLRPEPPGVQLEEKSSLQPKDIYPYSDGEWPPQASLSAGELTSPKSDSELEVRTPEPSPLRAESHMQWAWGRLPKVARAERPESSVVLEGRAGATSPPRGGPSTPSTSVAGGVDPLGLPIQQTEAGADLQPDTEDPTLVGPPLHTPETEESKTQSSGDMGLPPASKSWSWATLEVPVPTGQPERVSRGKGSPKRSQHLGPSDIYLDDLPSLDSENAALYFPQSDSGLGARRWSEPSSQKSLRDPNPEHEPEPTLDTVDTIALSLCGGLADSRDISLEKFNQHSVSYQDLTKNPGLLDDPNLVVKINGKHYNWAVAAPMILSLQAFQKNLPKSTMDKLEREKMPRKGGRWWFSWRRRDFLAEERSAQKEKTAAKEQQGEKTEVLSSDDDAPDSPVILEIPSLPPSTPPSTPTYKKSLRLSSDQIRRLNLQEGANDVVFSVTTQYQGTCRCKATIYLWKWDDKVVISDIDGTITKSDALGHILPQLGKDWTHQGITSLYHKIQLNGYKFLYCSARAIGMADLTKGYLQWVSEGGCSLPKGPILLSPSSLFSALHREVIEKKPEVFKVACLSDIQQLFLPHGQPFYAAFGNRPNDVFAYRQVGLPESRIFTVNPRGELIQELIKNHKSTYERLGEVVELLFPPVARGPSTDLANPEYSNFCYWREPLPAVDLDTLD",
# DPOG1_HUMAN
"MSRLLWRKVAGATVGPGPVPAPGRWVSSSVPASDPSDGQRRRQQQQQQQQQQQQQPQQPQVLSSEGGQLRHNPLDIQMLSRGLHEQIFGQGGEMPGEAAVRRSVEHLQKHGLWGQPAVPLPDVELRLPPLYGDNLDQHFRLLAQKQSLPYLEAANLLLQAQLPPKPPAWAWAEGWTRYGPEGEAVPVAIPEERALVFDVEVCLAEGTCPTLAVAISPSAWYSWCSQRLVEERYSWTSQLSPADLIPLEVPTGASSPTQRDWQEQLVVGHNVSFDRAHIREQYLIQGSRMRFLDTMSMHMAISGLSSFQRSLWIAAKQGKHKVQPPTKQGQKSQRKARRGPAISSWDWLDISSVNSLAEVHRLYVGGPPLEKEPRELFVKGTMKDIRENFQDLMQYCAQDVWATHEVFQQQLPLFLERCPHPVTLAGMLEMGVSYLPVNQNWERYLAEAQGTYEELQREMKKSLMDLANDACQLLSGERYKEDPWLWDLEWDLQEFKQKKAKKVKKEPATASKLPIEGAGAPGDPMDQEDLGPCSEEEEFQQDVMARACLQKLKGTTELLPKRPQHLPGHPGWYRKLCPRLDDPAWTPGPSLLSLQMRVTPKLMALTWDGFPLHYSERHGWGYLVPGRRDNLAKLPTGTTLESAGVVCPYRAIESLYRKHCLEQGKQQLMPQEAGLAEEFLLTDNSAIWQTVEELDYLEVEAEAKMENLRAAVPGQPLALTARGGPKDTQPSYHHGNGPYNDVDIPGCWFFKLPHKDGNSCNVGSPFAKDFLPKMEDGTLQAGPGGASGPRALEINKMISFWRNAHKRISSQMVVWLPRSALPRAVIRHPDYDEEGLYGAILPQVVTAGTITRRAVEPTWLTASNARPDRVGSELKAMVQAPPGYTLVGADVDSQELWIAAVLGDAHFAGMHGCTAFGWMTLQGRKSRGTDLHSKTATTVGISREHAKIFNYGRIYGAGQPFAERLLMQFNHRLTQQEAAEKAQQMYAATKGLRWYRLSDEGEWLVRELNLPVDRTEGGWISLQDLRKVQRETARKSQWKKWEVVAERAWKGGTESEMFNKLESIATSDIPRTPVLGCCISRALEPSAVQEEFMTSRVNWVVQSSAVDYLHLMLVAMKWLFEEFAIDGRFCISIHDEVRYLVREEDRYRAALALQITNLLTRCMFAYKLGLNDLPQSVAFFSAVDIDRCLRKEVTMDCKTPSNPTGMERRYGIPQGEALDIYQIIELTKGSLEKRSQPGP",
# MGDP1_HUMAN
"MARLPKLAVFDLDYTLWPFWVDTHVDPPFHKSSDGTVRDRRGQDVRLYPEVPEVLKRLQSLGVPGAAASRTSEIEGANQLLELFDLFRYFVHREIYPGSKITHFERLQQKTGIPFSQMIFFDDERRNIVDVSKLGVTCIHIQNGMNLQTLSQGLETFAKAQTGPLRSSLEESPFEA",
# 5NT3A_HUMAN
"MRAPSMDRAAVARVGAVASASVCALVAGVVLAQYIFTLKRKTGRKTKIIEMMPEFQKSSVRIKNPTRVEEIICGLIKGGAAKLQIITDFDMTLSRFSYKGKRCPTCHNIIDNCKLVTDECRKKLLQLKEKYYAIEVDPVLTVEEKYPYMVEWYTKSHGLLVQQALPKAKLKEIVAESDVMLKEGYENFFDKLQQHSIPVFIFSAGIGDVLEEVIRQAGVYHPNVKVVSNFMDFDETGVLKGFKGELIHVFNKHDGALRNTEYFNQLKDNSNIILLGDSQGDLRMADGVANVEHILKIGYLNDRVDELLEKYMDSYDIVLVQDESLEVANSILQKIL",
# 5NT3B_HUMAN
"MAEEVSTLMKATVLMRQPGRVQEIVGALRKGGGDRLQVISDFDMTLSRFAYNGKRCPSSYNILDNSKIISEECRKELTALLHHYYPIEIDPHRTVKEKLPHMVEWWTKAHNLLCQQKIQKFQIAQVVRESNAMLREGYKTFFNTLYHNNIPLFIFSAGIGDILEEIIRQMKVFHPNIHIVSNYMDFNEDGFLQGFKGQLIHTYNKNSSACENSGYFQQLEGKTNVILLGDSIGDLTMADGVPGVQNILKIGFLNDKVEERRERYMDSYDIVLEKDETLDVVNGLLQHILCQGVQLEMQGP",
# PHOP1_HUMAN
"MSGCFPVSGLRCLSRDGRMAAQGAPRFLLTFDFDETIVDENSDDSIVRAAPGQRLPESLRATYREGFYNEYMQRVFKYLGEQGVRPRDLSAIYEAIPLSPGMSDLLQFVAKQGACFEVILISDANTFGVESSLRAAGHHSLFRRILSNPSGPDARGLLALRPFHTHSCARCPANMCKHKVLSDYLRERAHDGVHFERLFYVGDGANDFCPMGLLAGGDVAFPRRGYPMHRLIQEAQKAEPSSFRASVVPWETAADVRLHLQQVLKSC",
# PHOP2_HUMAN
"MKILLVFDFDNTIIDDNSDTWIVQCAPNKKLPIELRDSYRKGFWTEFMGRVFKYLGDKGVREHEMKRAVTSLPFTPGMVELFNFIRKNKDKFDCIIISDSNSVFIDWVLEAASFHDIFDKVFTNPAAFNSNGHLTVENYHTHSCNRCPKNLCKKVVLIEFVDKQLQQGVNYTQIVYIGDGGNDVCPVTFLKNDDVAMPRKGYTLQKTLSRMSQNLEPMEYSVVVWSSGVDIISHLQFLIKD",
# SERB_HUMAN
"MVSHSELRKLFYSADAVCFDVDSTVIREEGIDELAKICGVEDAVSEMTRRAMGGAVPFKAALTERLALIQPSREQVQRLIAEQPPHLTPGIRELVSRLQERNVQVFLISGGFRSIVEHVASKLNIPATNVFANRLKFYFNGEYAGFDETQPTAESGGKGKVIKLLKEKFHFKKIIMIGDGATDMEACPPADAFIGFGGNVIRQQVKDNAKWYITDFVELLGELEE",
# PNKP_HUMAN
"MGEVEAPGRLWLESPPGGAPPIFLPSDGQALVLGRGPLTQVTDRKCSRTQVELVADPETRTVAVKQLGVNPSTTGTQELKPGLEGSLGVGDTLYLVNGLHPLTLRWEETRTPESQPDTPPGTPLVSQDEKRDAELPKKRMRKSNPGWENLEKLLVFTAAGVKPQGKVAGFDLDGTLITTRSGKVFPTGPSDWRILYPEIPRKLRELEAEGYKLVIFTNQMSIGRGKLPAEEFKAKVEAVVEKLGVPFQVLVATHAGLYRKPVTGMWDHLQEQANDGTPISIGDSIFVGDAAGRPANWAPGRKKKDFSCADRLFALNLGLPFATPEEFFLKWPAAGFELPAFDPRTVSRSGPLCLPESRALLSASPEVVVAVGFPGAGKSTFLKKHLVSAGYVHVNRDTLGSWQRCVTTCETALKQGKRVAIDNTNPDAASRARYVQCARAAGVPCRCFLFTATLEQARHNNRFREMTDSSHIPVSDMVMYGYRKQFEAPTLAEGFSAILEIPFRLWVEPRLGRLYCQFSEG",
# HYES_HUMAN
"MTLRAAVFDLDGVLALPAVFGVLGRTEEALALPRGLLNDAFQKGGPEGATTRLMKGEITLSQWIPLMEENCRKCSETAKVCLPKNFSIKEIFDKAISARKINRPMLQAALMLRKKGFTTAILTNTWLDDRAERDGLAQLMCELKMHFDFLIESCQVGMVKPEPQIYKFLLDTLKASPSEVVFLDDIGANLKPARDLGMVTILVQDTDTALKELEKVTGIQLLNTPAPLPTSCNPSDMSHGYVTVKPRVRLHFVELGSGPAVCLCHGFPESWYSWRYQIPALAQAGYRVLAMDMKGYGESSAPPEIEEYCMEVLCKEMVTFLDKLGLSQAVFIGHDWGGMLVWYMALFYPERVRAVASLNTPFIPANPNMSPLESIKANPVFDYQLYFQEPGVAEAELEQNLSRTFKSLFRASDESVLSMHKVCEAGGLFVNSPEEPSLSRMVTEEEIQFYVQQFKKSGFRGPLNWYRNMERNWKWACKSLGRKILIPALMVTAEKDFVLVPQMSQHMEDWIPHLKRGHIEDCGHWTQMDKPTEVNQILIKWLDSDARNPPVVSKM",
# HDDC2_HUMAN
"MASVSSATFSGHGARSLLQFLRLVGQLKRVPRTGWVYRNVQRPESVSDHMYRMAVMAMVIKDDRLNKDRCVRLALVHDMAECIVGDIAPADNIPKEEKHRREEEAMKQITQLLPEDLRKELYELWEEYETQSSAEAKFVKQLDQCEMILQASEYEDLEHKPGRLQDFYDSTAGKFNHPEIVQLVSELEAERSTNIAAAASEPHS",
# SAMH1_HUMAN
"MQRADSEQPSKRPRCDDSPRTPSNTPSAEADWSPGLELHPDYKTWGPEQVCSFLRRGGFEEPVLLKNIRENEITGALLPCLDESRFENLGVSSLGERKKLLSYIQRLVQIHVDTMKVINDPIHGHIELHPLLVRIIDTPQFQRLRYIKQLGGGYYVFPGASHNRFEHSLGVGYLAGCLVHALGEKQPELQISERDVLCVQIAGLCHDLGHGPFSHMFDGRFIPLARPEVKWTHEQGSVMMFEHLINSNGIKPVMEQYGLIPEEDICFIKEQIVGPLESPVEDSLWPYKGRPENKSFLYEIVSNKRNGIDVDKWDYFARDCHHLGIQNNFDYKRFIKFARVCEVDNELRICARDKEVGNLYDMFHTRNSLHRRAYQHKVGNIIDTMITDAFLKADDYIEITGAGGKKYRISTAIDDMEAYTKLTDNIFLEILYSTDPKLKDAREILKQIEYRNLFKYVGETQPTGQIKIKREDYESLPKEVASAKPKVLLDVKLKAEDFIVDVINMDYGMQEKNPIDHVSFYCKTAPNRAIRITKNQVSQLLPEKFAEQLIRVYCKKVDRKSLYAARQYFVQWCADRNFTKPQDGDVIAPLITPQKKEWNDSTSVQNPTRLREASKSRVQLFKDDPM",
# INP4A_HUMAN
"MTAREHSPRHGARARAMQRASTIDVAADMLGLSLAGNIQDPDEPILEFSLACSELHTPSLDRKPNSFVAVSVTTPPQAFWTKHAQTEIIEGTNNPIFLSSIAFFQDSLINQMTQVKLSVYDVKDRSQGTMYLLGSGTFIVKDLLQDRHHRLHLTLRSAESDRVGNITVIGWQMEEKSDQRPPVTRSVDTVNGRMVLPVDESLTEALGIRSKYASLRKDTLLKSVFGGAICRMYRFPTTDGNHLRILEQMAESVLSLHVPRQFVKLLLEEDAARVCELEELGELSPCWESLRRQIVTQYQTIILTYQENLTDLHQYRGPSFKASSLKADKKLEFVPTNLHIQRMRVQDDGGSDQNYDIVTIGAPAAHCQGFKSGGLRKKLHKFEETKKHFEECCTSSGCQSIIYIPQDVVRAKEIIAQINTLKTQVSYYAERLSRAAKDRSATGLERTLAILADKTRQLVTVCDCKLLANSIHGLNAARPDYIASKASPTSTEEEQVMLRNDQDTLMARWTGRNSRSSLQVDWHEEEWEKVWLNVDKSLECIIQRVDKLLQKERLHGEGCEDVFPCAGSCTSKKGNPDSHAYWIRPEDPFCDVPSSPCPSTMPSTACHPHLTTHCSPPPEESSPGEWSEALYPLLTTLTDCVAMMSDKAKKAMVFLLMQDSAPTIATYLSLQYRRDVVFCQTLTALICGFIIKLRNCLHDDGFLRQLYTIGLLAQFESLLSTYGEELAMLEDMSLGIMDLRNVTFKVTQATSSASADMLPVITGNRDGFNVRVPLPGPLFDALPREIQSGMLLRVQPVLFNVGINEQQTLAERFGDTSLQEVINVESLVRLNSYFEQFKEVLPEDCLPRSRSQTCLPELLRFLGQNVHARKNKNVDILWQAAEICRRLNGVRFTSCKSAKDRTAMSVTLEQCLILQHEHGMAPQVFTQALECMRSEGCRRENTMKNVGSRKYAFNSLQLKAFPKHYRPPEGTYGKVET",
# INP4B_HUMAN
"MEIKEEGASEEGQHFLPTAQANDPGDCQFTSIQKTPNEPQLEFILACKDLVAPVRDRKLNTLVQISVIHPVEQSLTRYSSTEIVEGTRDPLFLTGVTFPSEYPIYEETKIKLTVYDVKDKSHDTVRTSVLPEHKDPPPEVGRSFLGYASFKVGELLKSKEQLLVLSLRTSDGGKVVGTIEVSVVKMGEIEDGEADHITTDVQGQKCALVCECTAPESVSGKDNLPFLNSVLKNPVCKLYRFPTSDNKWMRIREQMSESILSFHIPKELISLHIKEDLCRNQEIKELGELSPHWDNLRKNVLTHCDQMVNMYQDILTELSKETGSSFKSSSSKGEKTLEFVPINLHLQRMQVHSPHLKDALYDVITVGAPAAHFQGFKNGGLRKLLHRFETERRNTGYQFIYYSPENTAKAKEVLSNINQLQPLIATHADLLLNSASQHSPDSLKNSLKMLSEKTELFVHAFKDQLVRSALLALYTARPGGILKKPPSPKSSTEESSPQDQPPVMRGQDSIPHHSDYDEEEWDRVWANVGKSLNCIIAMVDKLIERDGGSEGSGGNNDGEKEPSLTDAIPSHPREDWYEQLYPLILTLKDCMGEVVNRAKQSLTFVLLQELAYSLPQCLMLTLRRDIVFSQALAGLVCGFIIKLQTSLYDPGFLQQLHTVGLIVQYEGLLSTYSDEIGMLEDMAVGISDLKKVAFKIIEAKSNDVLPVITGRREHYVVEVKLPARMFESLPLQIKEGQLLHVYPVLFNVGINEQQTLAERFGDVSLQESINQENFELLQEYYKIFMEKMPPDYISHFQEQNDLKALLENLLQNIQSKKRKNVEIMWLAATICRKLNGIRFTCCKSAKDRTSMSVTLEQCSILRDEHQLHKDFFIRALDCMRREGCRIENVLKNIKCRKYAFNMLQLMAFPKYYRPPEGTYGKADT",
# NUD10_HUMAN
"MKCKPNQTRTYDPEGFKKRAACLCFRSEREDEVLLVSSSRYPDRWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGVFEQNQDPKHRTYVYVLTVTELLEDWEDSVSIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGGSPTNGNSMAPSSPDSDP",
# NUDT9_HUMAN
"MAGRLLGKALAAVSLSLALASVTIRSSRCRGIQAFRNSFSSSWFHLNTNVMSGSNGSKENSHNKARTSPYPGSKVERSQVPNEKVGWLVEWQDYKPVEYTAVSVLAGPRWADPQISESNFSPKFNEKDGHVERKSKNGLYEIENGRPRNPAGRTGLVGRGLLGRWGPNHAADPIITRWKRDSSGNKIMHPVSGKHILQFVAIKRKDCGEWAIPGGMVDPGEKISATLKREFGEEALNSLQKTSAEKREIEEKLHKLFSQDHLVIYKGYVDDPRNTDNAWMETEAVNYHDETGEIMDNLMLEAGDDAGKVKWVDINDKLKLYASHSQFIKLVAEKRDAHWSEDSEADCHAL",
# NUD11_HUMAN
"MKCKPNQTRTYDPEGFKKRAACLCFRSEREDEVLLVSSSRYPDRWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGVFEQNQDRKHRTYVYVLTVTELLEDWEDSVSIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGGSPTNGNSMAPSSPDSDP",
# NUD14_HUMAN
"MERIEGASVGRCAASPYLRPLTLHYRQNGAQKSWDFMKTHDSVTVLLFNSSRRSLVLVKQFRPAVYAGEVERRFPGSLAAVDQDGPRELQPALPGSAGVTVELCAGLVDQPGLSLEEVACKEAWEECGYHLAPSDLRRVATYWSGVGLTGSRQTMFYTEVTDAQRSGPGGGLVEEGELIEVVHLPLEGAQAFADDPDIPKTLGVIFGVSWFLSQVAPNLDLQ",
# NUDT3_HUMAN
"MMKLKSNQTRTYDGDGYKKRAACLCFRSESEEEVLLVSSSRHPDRWIVPGGGMEPEEEPSVAAVREVCEEAGVKGTLGRLVGIFENQERKHRTYVYVLIVTEVLEDWEDSVNIGRKREWFKIEDAIKVLQYHKPVQASYFETLRQGYSANNGTPVVATTYSVSAQSSMSGIR",
# NUDT4_HUMAN
"MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGIFENQDRKHRTYVYVLTVTEILEDWEDSVNIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGCSPANGNSTVPSLPDNNALFVTAAQTSGLPSSVR",
# IPYR_HUMAN
"MSGFSTEERAAPFSLEYRVFLKNEKGQYISPFHDIPIYADKDVFHMVVEVPRWSNAKMEIATKDPLNPIKQDVKKGKLRYVANLFPYKGYIWNYGAIPQTWEDPGHNDKHTGCCGDNDPIDVCEIGSKVCARGEIIGVKVLGILAMIDEGETDWKVIAINVDDPDAANYNDINDVKRLKPGYLEATVDWFRRYKVPDGKPENEFAFNAEFKDKDFAIDIIKSTHDHWKALVTKKTNGKGISCMNTTLSESPFKCDPDAARAIVDALPPPCESACTVPTDVDKWFHHQKN",
# IPYR2_HUMAN
"MSALLRLLRTGAPAAACLRLGTSAGTGSRRAMALYHTEERGQPCSQNYRLFFKNVTGHYISPFHDIPLKVNSKEENGIPMKKARNDEYENLFNMIVEIPRWTNAKMEIATKEPMNPIKQYVKDGKLRYVANIFPYKGYIWNYGTLPQTWEDPHEKDKSTNCFGDNDPIDVCEIGSKILSCGEVIHVKILGILALIDEGETDWKLIAINANDPEASKFHDIDDVKKFKPGYLEATLNWFRLYKVPDGKPENQFAFNGEFKNKAFALEVIKSTHQCWKALLMKKCNGGAINCTNVQISDSPFRCTQEEARSLVESVSSSPNKESNEEEQVWHFLGK",
# 5NTD_HUMAN
"MCPRAARAPATLLLALGAVLWPAAGAWELTILHTNDVHSRLEQTSEDSSKCVNASRCMGGVARLFTKVQQIRRAEPNVLLLDAGDQYQGTIWFTVYKGAEVAHFMNALRYDAMALGNHEFDNGVEGLIEPLLKEAKFPILSANIKAKGPLASQISGLYLPYKVLPVGDEVVGIVGYTSKETPFLSNPGTNLVFEDEITALQPEVDKLKTLNVNKIIALGHSGFEMDKLIAQKVRGVDVVVGGHSNTFLYTGNPPSKEVPAGKYPFIVTSDDGRKVPVVQAYAFGKYLGYLKIEFDERGNVISSHGNPILLNSSIPEDPSIKADINKWRIKLDNYSTQELGKTIVYLDGSSQSCRFRECNMGNLICDAMINNNLRHTDEMFWNHVSMCILNGGGIRSPIDERNNGTITWENLAAVLPFGGTFDLVQLKGSTLKKAFEHSVHRYGQSTGEFLQVGGIHVVYDLSRKPGDRVVKLDVLCTKCRVPSYDPLKMDEVYKVILPNFLANGGDGFQMIKDELLRHDSGDQDINVVSTYISKMKVIYPAVEGRIKFSTGSHCHGSFSLIFLSLWAVIFVLYQ",
# VPS29_HUMAN
"MLVLVLGDLHIPHRCNSLPAKFKKLLVPGKIQHILCTGNLCTKESYDYLKTLAGDVHIVRGDFDENLNYPEQKVVTVGQFKIGLIHGHQVIPWGDMASLALLQRQFDVDILISGHTHKFEAFEHENKFYINPGSATGAYNALETNIIPSFVLMDIQASTVVTYVYQLIGDDVKVERIEYKKP",
# PP4P2_HUMAN
"MAADGVDERSPLLSASHSGNVTPTAPPYLQESSPRAELPPPYTAIASPDASGIPVINCRVCQSLINLDGKLHQHVVKCTVCNEATPIKNPPTGKKYVRCPCNCLLICKDTSRRIGCPRPNCRRIINLGPVMLISEEQPAQPALPIQPEGTRVVCGHCGNTFLWMELRFNTLAKCPHCKKISSVGSALPRRRCCAYITIGMICIFIGVGLTVGTPDFARRFRATYVSWAIAYLLGLICLIRACYWGAIRVSYPEHSFA",
# PP4P1_HUMAN
"MAADGERSPLLSEPIDGGAGGNGLVGPGGSGAGPGGGLTPSAPPYGAAFPPFPEGHPAVLPGEDPPPYSPLTSPDSGSAPMITCRVCQSLINVEGKMHQHVVKCGVCNEATPIKNAPPGKKYVRCPCNCLLICKVTSQRIACPRPYCKRIINLGPVHPGPLSPEPQPMGVRVICGHCKNTFLWTEFTDRTLARCPHCRKVSSIGRRYPRKRCICCFLLGLLLAVTATGLAFGTWKHARRYGGIYAAWAFVILLAVLCLGRALYWACMKVSHPVQNFS"
]


# Add protein names corresponding to the sequences
protein_names = [
"PPBI_HUMAN",
"PPBT_HUMAN",
"PPB1_HUMAN",
"PPBN_HUMAN",
"CC14A_HUMAN",
"CC14B_HUMAN",
"CDK3_HUMAN",
"CDKN3_HUMAN",
"DUS1_HUMAN",
"DUSP2_HUMAN",
"DUS4_HUMAN",
"DUS5_HUMAN",
"DUS10_HUMAN",
"DUS11_HUMAN",
"DUS12_HUMAN",
"DUS14_HUMAN",
"DUS18_HUMAN",
"DUS21_HUMAN",
"DUS28_HUMAN",
"DUS15_HUMAN",
"DUS22_HUMAN",
"DUS19_HUMAN",
"DUS23_HUMAN",
"DUS29_HUMAN",
"DS13A_HUMAN",
"DS13B_HUMAN",
"DUS26_HUMAN",
"STYL2_HUMAN",
"DUS3_HUMAN",
"DUS6_HUMAN",
"DUS7_HUMAN",
"DUS9_HUMAN",
"DUS16_HUMAN",
"DUS8_HUMAN",
"TP4A1_HUMAN",
"TP4A2_HUMAN",
"TP4A3_HUMAN",
"PTPC1_HUMAN",
"PTPM1_HUMAN",
"MCE1_HUMAN",
"SSH1_HUMAN",
"SSH2_HUMAN",
"SSH3_HUMAN",
"STYX_HUMAN",
"STYL1_HUMAN",
"MTM1_HUMAN",
"MTMR1_HUMAN",
"MTMR2_HUMAN",
"MTMRA_HUMAN",
"MTMRB_HUMAN",
"MTMRC_HUMAN",
"MTMRE_HUMAN",
"MTMR3_HUMAN",
"MTMR4_HUMAN",
"MTMR5_HUMAN",
"MTMRD_HUMAN",
"MTMR6_HUMAN",
"MTMR7_HUMAN",
"MTMR8_HUMAN",
"MTMR9_HUMAN",
"PALD_HUMAN",
"AUXI_HUMAN",
"GAK_HUMAN",
"PTEN_HUMAN",
"TENS1_HUMAN",
"TENS2_HUMAN",
"TENS3_HUMAN",
"TPTE_HUMAN",
"TPTE2_HUMAN",
"PTN1_HUMAN",
"PTN2_HUMAN",
"PTN12_HUMAN",
"PTN18_HUMAN",
"PTN22_HUMAN",
"PTN13_HUMAN",
"PTN20_HUMAN",
"PTN14_HUMAN",
"PTN21_HUMAN",
"PTN23_HUMAN",
"PTN3_HUMAN",
"PTN4_HUMAN",
"PTN5_HUMAN",
"PTN7_HUMAN",
"PTPRR_HUMAN",
"PTN11_HUMAN",
"PTN6_HUMAN",
"PTN9_HUMAN",
"PTPRA_HUMAN",
"PTPRE_HUMAN",
"PTPRB_HUMAN",
"PTPRH_HUMAN",
"PTPRJ_HUMAN",
"PTPRO_HUMAN",
"PTPRU_HUMAN",
"PTPRQ_HUMAN",
"PTPRC_HUMAN",
"PTPRD_HUMAN",
"PTPRF_HUMAN",
"PTPRS_HUMAN",
"PTPRG_HUMAN",
"PTPRZ_HUMAN",
"PTPRK_HUMAN",
"PTPRM_HUMAN",
"PTPRT_HUMAN",
"PTPRN_HUMAN",
"PTPR2_HUMAN",
"FIG4_HUMAN",
"SAC2_HUMAN",
"SAC1_HUMAN",
"SYNJ1_HUMAN",
"SYNJ2_HUMAN",
"SSU72_HUMAN",
"MPIP1_HUMAN",
"MPIP2_HUMAN",
"MPIP3_HUMAN",
"EYA1_HUMAN",
"EYA2_HUMAN",
"EYA3_HUMAN",
"EYA4_HUMAN",
"CTSL2_HUMAN",
"CNEP1_HUMAN",
"CTDP1_HUMAN",
"SYCP1_HUMAN",
"CTDS1_HUMAN",
"RISC_HUMAN",
"CTDS2_HUMAN",
"SCP2_HUMAN",
"SYCP2_HUMAN",
"CTDSL_HUMAN",
"SYCP3_HUMAN",
"TIM50_HUMAN",
"UBCP1_HUMAN",
"HDHD5_HUMAN",
"HDHD2_HUMAN",
"LHPP_HUMAN",
"PLPP_HUMAN",
"PGP_HUMAN",
"F261_HUMAN",
"F262_HUMAN",
"F263_HUMAN",
"F264_HUMAN",
"PMGE_HUMAN",
"PGAM1_HUMAN",
"PGAM2_HUMAN",
"PGAM4_HUMAN",
"PGAM5_HUMAN",
"UBS3A_HUMAN",
"UBS3B_HUMAN",
"TIGAR_HUMAN",
"PPAL_HUMAN",
"PPAP_HUMAN",
"PPAT_HUMAN",
"PPA6_HUMAN",
"MINP1_HUMAN",
"VIP1_HUMAN",
"VIP2_HUMAN",
"PXYP1_HUMAN",
"ILKAP_HUMAN",
"PLPP6_HUMAN",
"PDP1_HUMAN",
"PDP2_HUMAN",
"PHLP1_HUMAN",
"PHLP2_HUMAN",
"PP2D1_HUMAN",
"PPM1A_HUMAN",
"PPM1B_HUMAN",
"PPM1N_HUMAN",
"PPM1D_HUMAN",
"PPM1E_HUMAN",
"PPM1M_HUMAN",
"PPM1F_HUMAN",
"PPM1G_HUMAN",
"PPM1H_HUMAN",
"PPM1J_HUMAN",
"PPM1K_HUMAN",
"PPM1L_HUMAN",
"PPTC7_HUMAN",
"TAB1_HUMAN",
"PPA5_HUMAN",
"ACP7_HUMAN",
"PP1A_HUMAN",
"PP1B_HUMAN",
"PP1G_HUMAN",
"PP2AA_HUMAN",
"PP2AB_HUMAN",
"PP2BA_HUMAN",
"PP2BB_HUMAN",
"PP2BC_HUMAN",
"PP4C_HUMAN",
"PPP5_HUMAN",
"PPP6_HUMAN",
"PPE1_HUMAN",
"PPE2_HUMAN",
"RPAP2_HUMAN",
"ITPA_HUMAN",
"DOPP1_HUMAN",
"G6PC1_HUMAN",
"G6PC2_HUMAN",
"G6PC3_HUMAN",
"PLPR1_HUMAN",
"PLPR2_HUMAN",
"PLPR3_HUMAN",
"PLPR4_HUMAN",
"PLPR5_HUMAN",
"PLPP1_HUMAN",
"PLPP3_HUMAN",
"PLPP2_HUMAN",
"PLPP4_HUMAN",
"PLPP5_HUMAN",
"PLPP7_HUMAN",
"SGPP1_HUMAN",
"SGPP2_HUMAN",
"BPNT1_HUMAN",
"F16P1_HUMAN",
"F16P2_HUMAN",
"PCX4_HUMAN",
"IMPA1_HUMAN",
"IMPA2_HUMAN",
"IMPA3_HUMAN",
"INPP_HUMAN",
"I5P1_HUMAN",
"I5P2_HUMAN",
"SHIP1_HUMAN",
"MPPA_HUMAN",
"INP5E_HUMAN",
"PI5PA_HUMAN",
"INP5K_HUMAN",
"SHIP2_HUMAN",
"OCRL_HUMAN",
"5NT1A_HUMAN",
"5NT1B_HUMAN",
"5NTC_HUMAN",
"NT5D1_HUMAN",
"NT5D2_HUMAN",
"NT5D3_HUMAN",
"NT5D4_HUMAN",
"PMM1_HUMAN",
"PMM2_HUMAN",
"NT5C_HUMAN",
"NT5M_HUMAN",
"ENOPH_HUMAN",
"HDHD3_HUMAN",
"NANP_HUMAN",
"HDHD1_HUMAN",
"LPIN1_HUMAN",
"LPIN2_HUMAN",
"LPIN3_HUMAN",
"DPOG1_HUMAN",
"MGDP1_HUMAN",
"5NT3A_HUMAN",
"5NT3B_HUMAN",
"PHOP1_HUMAN",
"PHOP2_HUMAN",
"SERB_HUMAN",
"PNKP_HUMAN",
"HYES_HUMAN",
"HDDC2_HUMAN",
"SAMH1_HUMAN",
"INP4A_HUMAN",
"INP4B_HUMAN",
"NUD10_HUMAN",
"NUDT9_HUMAN",
"NUD11_HUMAN",
"NUD14_HUMAN",
"NUDT3_HUMAN",
"NUDT4_HUMAN",
"IPYR_HUMAN",
"IPYR2_HUMAN",
"5NTD_HUMAN",
"VPS29_HUMAN",
"PP4P2_HUMAN",
"PP4P1_HUMAN"

]

In [39]:
#Proteasome
sequences = [
# PSA1_HUMAN
"MFRNQYDNDVTVWSPQGRIHQIEYAMEAVKQGSATVGLKSKTHAVLVALKRAQSELAAHQKKILHVDNHIGISIAGLTADARLLCNFMRQECLDSRFVFDRPLPVSRLVSLIGSKTQIPTQRYGRRPYGVGLLIAGYDDMGPHIFQTCPSANYFDCRAMSIGARSQSARTYLERHMSEFMECNLNELVKHGLRALRETLPAEQDLTTKNVSIGIVGKDLEFTIYDDDDVSPFLEGLEERPQRKAQPAQPADEPAEKADEPMEH",
# PSA2_HUMAN
"MAERGYSFSLTTFSPSGKLVQIEYALAAVAGGAPSVGIKAANGVVLATEKKQKSILYDERSVHKVEPITKHIGLVYSGMGPDYRVLVHRARKLAQQYYLVYQEPIPTAQLVQRVASVMQEYTQSGGVRPFGVSLLICGWNEGRPYLFQSDPSGAYFAWKATAMGKNYVNGKTFLEKRYNEDLELEDAIHTAILTLKESFEGQMTEDNIEVGICNEAGFRRLTPTEVKDYLAAIA",
# PSA3_HUMAN
"MSSIGTGYDLSASTFSPDGRVFQVEYAMKAVENSSTAIGIRCKDGVVFGVEKLVLSKLYEEGSNKRLFNVDRHVGMAVAGLLADARSLADIAREEASNFRSNFGYNIPLKHLADRVAMYVHAYTLYSAVRPFGCSFMLGSYSVNDGAQLYMIDPSGVSYGYWGCAIGKARQAAKTEIEKLQMKEMTCRDIVKEVAKIIYIVHDEVKDKAFELELSWVGELTNGRHEIVPKDIREEAEKYAKESLKEEDESDDDNM",
# PSA4_HUMAN
"MSRRYDSRTTIFSPEGRLYQVEYAMEAIGHAGTCLGILANDGVLLAAERRNIHKLLDEVFFSEKIYKLNEDMACSVAGITSDANVLTNELRLIAQRYLLQYQEPIPCEQLVTALCDIKQAYTQFGGKRPFGVSLLYIGWDKHYGFQLYQSDPSGNYGGWKATCIGNNSAAAVSMLKQDYKEGEMTLKSALALAIKVLNKTMDVSKLSAEKVEIATLTRENGKTVIRVLKQKEVEQLIKKHEEEEAKAEREKKEKEQKEKDK",
# PSA5_HUMAN
"MFLTRSEYDRGVNTFSPEGRLFQVEYAIEAIKLGSTAIGIQTSEGVCLAVEKRITSPLMEPSSIEKIVEIDAHIGCAMSGLIADAKTLIDKARVETQNHWFTYNETMTVESVTQAVSNLALQFGEEDADPGAMSRPFGVALLFGGVDEKGPQLFHMDPSGTFVQCDARAIGSASEGAQSSLQEVYHKSMTLKEAIKSSLIILKQVMEEKLNATNIELATVQPGQNFHMFTKEELEEVIKDI",
# PSA6_HUMAN
"MSRGSSAGFDRHITIFSPEGRLYQVEYAFKAINQGGLTSVAVRGKDCAVIVTQKKVPDKLLDSSTVTHLFKITENIGCVMTGMTADSRSQVQRARYEAANWKYKYGYEIPVDMLCKRIADISQVYTQNAEMRPLGCCMILIGIDEEQGPQVYKCDPAGYYCGFKATAAGVKQTESTSFLEKKVKKKFDWTFEQTVETAITCLSTVLSIDFKPSEIEVGVVTVENPKFRILTEAEIDAHLVALAERD",
# PSA7_HUMAN
"MSYDRAITVFSPDGHLFQVEYAQEAVKKGSTAVGVRGRDIVVLGVEKKSVAKLQDERTVRKICALDDNVCMAFAGLTADARIVINRARVECQSHRLTVEDPVTVEYITRYIASLKQRYTQSNGRRPFGISALIVGFDFDGTPRLYQTDPSGTYHAWKANAIGRGAKSVREFLEKNYTDEAIETDDLTIKLVIKALLEVVQSGGKNIELAVMRRDQSLKILNPEEIEKYVAEIEKEKEENEKKKQKKAS",
# PSMA8_HUMAN
"MASRYDRAITVFSPDGHLFQVEYAQEAVKKGSTAVGIRGTNIVVLGVEKKSVAKLQDERTVRKICALDDHVCMAFAVLTIFIGLTADARVVINRARVECQSHKLTVEDPVTVEYITRFIATLKQKYTQSNGRRPFGISALIVGFDDDGISRLYQTDPSGTYHAWKANAIGRSAKTVREFLEKNYTEDAIASDSEAIKLAIKALLEVVQSGGKNIELAIIRRNQPLKMFSAKEVELYVTEIEKEKEEAEKKKSKKSV",
# PSB1_HUMAN
"MLSSTAMYSAPGRDLGMEPHRAAGPLQLRFSPYVFNGGTILAIAGEDFAIVASDTRLSEGFSIHTRDSPKCYKLTDKTVIGCSGFHGDCLTLTKIIEARLKMYKHSNNKAMTTGAIAAMLSTILYSRRFFPYYVYNIIGGLDEEGKGAVYSFDPVGSYQRDSFKAGGSASAMLQPLLDNQVGFKNMQNVEHVPLSLDRAMRLVKDVFISAAERDVYTGDALRICIVTKEGIREETVSLRKD",
# PSB2_HUMAN
"MEYLIGIQGPDYVLVASDRVAASNIVQMKDDHDKMFKMSEKILLLCVGEAGDTVQFAEYIQKNVQLYKMRNGYELSPTAAANFTRRNLADCLRSRTPYHVNLLLAGYDEHEGPALYYMDYLAALAKAPFAAHGYGAFLTLSILDRYYTPTISRERAVELLRKCLEELQKRFILNLPTFSVRIIDKNGIHDLDNISFPKQGS",
# PSB3_HUMAN
"MSIMSYNGGAVMAMKGKNCVAIAADRRFGIQAQMVTTDFQKIFPMGDRLYIGLAGLATDVQTVAQRLKFRLNLYELKEGRQIKPYTLMSMVANLLYEKRFGPYYTEPVIAGLDPKTFKPFICSLDLIGCPMVTDDFVVSGTCAEQMYGMCESLWEPNMDPDHLFETISQAMLNAVDRDAVSGMGVIVHIIEKDKITTRTLKARMD",
# PSB4_HUMAN
"MEAFLGSRSGLWAGGPAPGQFYRIPSTPDSFMDPASALYRGPITRTQNPMVTGTSVLGVKFEGGVVIAADMLGSYGSLARFRNISRIMRVNNSTMLGASGDYADFQYLKQVLGQMVIDEELLGDGHSYSPRAIHSWLTRAMYSRRSKMNPLWNTMVIGGYADGESFLGYVDMLGVAYEAPSLATGYGAYLAQPLLREVLEKQPVLSQTEARDLVERCMRVLYYRDARSYNRFQIATVTEKGVEIEGPLSTETNWDIAHMISGFE",
# PSB5_HUMAN
"MALASVLERPLPVNQRGFFGLGGRADLLDLGPGSLSDGLSLAAPGWGVPEEPGIEMLHGTTTLAFKFRHGVIVAADSRATAGAYIASQTVKKVIEINPYLLGTMAGGAADCSFWERLLARQCRIYELRNKERISVAAASKLLANMVYQYKGMGLSMGTMICGWDKRGPGLYYVDSEGNRISGATFSVGSGSVYAYGVMDRGYSYDLEVEQAYDLARRAIYQATYRDAYSGGAVNLYHVREDGWIRVSSDNVADLHEKYSGSTP",
# PSB6_HUMAN
"MAATLLAARGAGPAPAWGPEAFTPDWESREVSTGTTIMAVQFDGGVVLGADSRTTTGSYIANRVTDKLTPIHDRIFCCRSGSAADTQAVADAVTYQLGFHSIELNEPPLVHTAASLFKEMCYRYREDLMAGIIIAGWDPQEGGQVYSVPMGGMMVRQSFAIGGSGSSYIYGYVDATYREGMTKEECLQFTANALALAMERDGSSGGVIRLAAIAESGVERQVLLGDQIPKFAVATLPPA",
# PSB7_HUMAN
"MAAVSVYAPPVGGFSFDNCRRNAVLEADFAKRGYKLPKVRKTGTTIAGVVYKDGIVLGADTRATEGMVVADKNCSKIHFISPNIYCCGAGTAADTDMTTQLISSNLELHSLSTGRLPRVVTANRMLKQMLFRYQGYIGAALVLGGVDVTGPHLYSIYPHGSTDKLPYVTMGSGSLAAMAVFEDKFRPDMEEEEAKNLVSEAIAAGIFNDLGSGSNIDLCVISKNKLDFLRPYTVPNKKGTRLGRYRCEKGTTAVLTEKITPLEIEVLEETVQTMDTS",
# PSB8_HUMAN
"MALLDVCGAPRGQRPESALPVAGSGRRSDPGHYSFSMRSPELALPRGMQPTEFFQSLGGDGERNVQIEMAHGTTTLAFKFQHGVIAAVDSRASAGSYISALRVNKVIEINPYLLGTMSGCAADCQYWERLLAKECRLYYLRNGERISVSAASKLLSNMMCQYRGMGLSMGSMICGWDKKGPGLYYVDEHGTRLSGNMFSTGSGNTYAYGVMDSGYRPNLSPEEAYDLGRRAIAYATHRDSYSGGVVNMYHMKEDGWVKVESTDVSDLLHQYREANQ",
# PSB9_HUMAN
"MLRAGAPTGDLPRAGEVHTGTTIMAVEFDGGVVMGSDSRVSAGEAVVNRVFDKLSPLHERIYCALSGSAADAQAVADMAAYQLELHGIELEEPPLVLAAANVVRNISYKYREDLSAHLMVAGWDQREGGQVYGTLGGMLTRQPFAIGGSGSTFIYGYVDAAYKPGMSPEECRRFTTDAIALAMSRDGSSGGVIYLVTITAAGVDHRVILGNELPKFYDE",
# PSB10_HUMAN
"MLKPALEPRGGFSFENCQRNASLERVLPGLKVPHARKTGTTIAGLVFQDGVILGADTRATNDSVVADKSCEKIHFIAPKIYCCGAGVAADAEMTTRMVASKMELHALSTGREPRVATVTRILRQTLFRYQGHVGASLIVGGVDLTGPQLYGVHPHGSYSRLPFTALGSGQDAALAVLEDRFQPNMTLEAAQGLLVEAVTAGILGDLGSGGNVDACVITKTGAKLLRTLSSPTEPVKRSGRYHFVPGTTAVLTQTVKPLTLELVEETVQAMEVE",
# PRS4_HUMAN
"MGQSQSGGHGPGGGKKDDKDKKKKYEPPVPTRVGKKKKKTKGPDAASKLPLVTPHTQCRLKLLKLERIKDYLLMEEEFIRNQEQMKPLEEKQEEERSKVDDLRGTPMSVGTLEEIIDDNHAIVSTSVGSEHYVSILSFVDKDLLEPGCSVLLNHKVHAVIGVLMDDTDPLVTVMKVEKAPQETYADIGGLDNQIQEIKESVELPLTHPEYYEEMGIKPPKGVILYGPPGTGKTLLAKAVANQTSATFLRVVGSELIQKYLGDGPKLVRELFRVAEEHAPSIVFIDEIDAIGTKRYDSNSGGEREIQRTMLELLNQLDGFDSRGDVKVIMATNRIETLDPALIRPGRIDRKIEFPLPDEKTKKRIFQIHTSRMTLADDVTLDDLIMAKDDLSGADIKAICTEAGLMALRERRMKVTNEDFKKSKENVLYKKQEGTPEGLYL",
# PRS7_HUMAN
"MPDYLGADQRKTKEDEKDDKPIRALDEGDIALLKTYGQSTYSRQIKQVEDDIQQLLKKINELTGIKESDTGLAPPALWDLAADKQTLQSEQPLQVARCTKIINADSEDPKYIINVKQFAKFVVDLSDQVAPTDIEEGMRVGVDRNKYQIHIPLPPKIDPTVTMMQVEEKPDVTYSDVGGCKEQIEKLREVVETPLLHPERFVNLGIEPPKGVLLFGPPGTGKTLCARAVANRTDACFIRVIGSELVQKYVGEGARMVRELFEMARTKKACLIFFDEIDAIGGARFDDGAGGDNEVQRTMLELINQLDGFDPRGNIKVLMATNRPDTLDPALMRPGRLDRKIEFSLPDLEGRTHIFKIHARSMSVERDIRFELLARLCPNSTGAEIRSVCTEAGMFAIRARRKIATEKDFLEAVNKVIKSYAKFSATPRYMTYN",
# PRS6A_HUMAN
"MNLLPNIESPVTRQEKMATVWDEAEQDGIGEEVLKMSTEEIIQRTRLLDSEIKIMKSEVLRVTHELQAMKDKIKENSEKIKVNKTLPYLVSNVIELLDVDPNDQEEDGANIDLDSQRKGKCAVIKTSTRQTYFLPVIGLVDAEKLKPGDLVGVNKDSYLILETLPTEYDSRVKAMEVDERPTEQYSDIGGLDKQIQELVEAIVLPMNHKEKFENLGIQPPKGVLMYGPPGTGKTLLARACAAQTKATFLKLAGPQLVQMFIGDGAKLVRDAFALAKEKAPSIIFIDELDAIGTKRFDSEKAGDREVQRTMLELLNQLDGFQPNTQVKVIAATNRVDILDPALLRSGRLDRKIEFPMPNEEARARIMQIHSRKMNVSPDVNYEELARCTDDFNGAQCKAVCVEAGMIALRRGATELTHEDYMEGILEVQAKKKANLQYYA",
# PRS6B_HUMAN
"MEEIGILVEKAQDEIPALSVSRPQTGLSFLGPEPEDLEDLYSRYKKLQQELEFLEVQEEYIKDEQKNLKKEFLHAQEEVKRIQSIPLVIGQFLEAVDQNTAIVGSTTGSNYYVRILSTIDRELLKPNASVALHKHSNALVDVLPPEADSSIMMLTSDQKPDVMYADIGGMDIQKQEVREAVELPLTHFELYKQIGIDPPRGVLMYGPPGCGKTMLAKAVAHHTTAAFIRVVGSEFVQKYLGEGPRMVRDVFRLAKENAPAIIFIDEIDAIATKRFDAQTGADREVQRILLELLNQMDGFDQNVNVKVIMATNRADTLDPALLRPGRLDRKIEFPLPDRRQKRLIFSTITSKMNLSEEVDLEDYVARPDKISGADINSICQESGMLAVRENRYIVLAKDFEKAYKTVIKKDEQEHEFYK",
# PRS8_HUMAN
"MALDGPEQMELEEGKAGSGLRQYYLSKIEELQLIVNDKSQNLRRLQAQRNELNAKVRLLREELQLLQEQGSYVGEVVRAMDKKKVLVKVHPEGKFVVDVDKNIDINDVTPNCRVALRNDSYTLHKILPNKVDPLVSLMMVEKVPDSTYEMIGGLDKQIKEIKEVIELPVKHPELFEALGIAQPKGVLLYGPPGTGKTLLARAVAHHTDCTFIRVSGSELVQKFIGEGARMVRELFVMAREHAPSIIFMDEIDSIGSSRLEGGSGGDSEVQRTMLELLNQLDGFEATKNIKVIMATNRIDILDSALLRPGRIDRKIEFPPPNEEARLDILKIHSRKMNLTRGINLRKIAELMPGASGAEVKGVCTEAGMYALRERRVHVTQEDFEMAVAKVMQKDSEKNMSIKKLWK",
# PRS10_HUMAN
"MADPRDKALQDYRKKLLEHKEIDGRLKELREQLKELTKQYEKSENDLKALQSVGQIVGEVLKQLTEEKFIVKATNGPRYVVGCRRQLDKSKLKPGTRVALDMTTLTIMRYLPREVDPLVYNMSHEDPGNVSYSEIGGLSEQIRELREVIELPLTNPELFQRVGIIPPKGCLLYGPPGTGKTLLARAVASQLDCNFLKVVSSSIVDKYIGESARLIREMFNYARDHQPCIIFMDEIDAIGGRRFSEGTSADREIQRTLMELLNQMDGFDTLHRVKMIMATNRPDTLDPALLRPGRLDRKIHIDLPNEQARLDILKIHAGPITKHGEIDYEAIVKLSDGFNGADLRNVCTEAGMFAIRADHDFVVQEDFMKAVRKVADSKKLESKLDYKPV",
# PSMD1_HUMAN
"MITSAAGIISLLDEDEPQLKEFALHKLNAVVNDFWAEISESVDKIEVLYEDEGFRSRQFAALVASKVFYHLGAFEESLNYALGAGDLFNVNDNSEYVETIIAKCIDHYTKQCVENADLPEGEKKPIDQRLEGIVNKMFQRCLDDHKYKQAIGIALETRRLDVFEKTILESNDVPGMLAYSLKLCMSLMQNKQFRNKVLRVLVKIYMNLEKPDFINVCQCLIFLDDPQAVSDILEKLVKEDNLLMAYQICFDLYESASQQFLSSVIQNLRTVGTPIASVPGSTNTGTVPGSEKDSDSMETEEKTSSAFVGKTPEASPEPKDQTLKMIKILSGEMAIELHLQFLIRNNNTDLMILKNTKDAVRNSVCHTATVIANSFMHCGTTSDQFLRDNLEWLARATNWAKFTATASLGVIHKGHEKEALQLMATYLPKDTSPGSAYQEGGGLYALGLIHANHGGDIIDYLLNQLKNASNDIVRHGGSLGLGLAAMGTARQDVYDLLKTNLYQDDAVTGEAAGLALGLVMLGSKNAQAIEDMVGYAQETQHEKILRGLAVGIALVMYGRMEEADALIESLCRDKDPILRRSGMYTVAMAYCGSGNNKAIRRLLHVAVSDVNDDVRRAAVESLGFILFRTPEQCPSVVSLLSESYNPHVRYGAAMALGICCAGTGNKEAINLLEPMTNDPVNYVRQGALIASALIMIQQTEITCPKVNQFRQLYSKVINDKHDDVMAKFGAILAQGILDAGGHNVTISLQSRTGHTHMPSVVGVLVFTQFWFWFPLSHFLSLAYTPTCVIGLNKDLKMPKVQYKSNCKPSTFAYPAPLEVPKEKEKEKVSTAVLSITAKAKKKEKEKEKKEEEKMEVDEAEKKEEKEKKKEPEPNFQLLDNPARVMPAQLKVLTMPETCRYQPFKPLSIGGIIILKDTSEDIEELVEPVAAHGPKIEEEEQEPEPPEPFEYIDD",
# PSMD2_HUMAN
"MEEGGRDKAPVQPQQSPAAAPGGTDEKPSGKERRDAGDKDKEQELSEEDKQLQDELEMLVERLGEKDTSLYRPALEELRRQIRSSTTSMTSVPKPLKFLRPHYGKLKEIYENMAPGENKRFAADIISVLAMTMSGERECLKYRLVGSQEELASWGHEYVRHLAGEVAKEWQELDDAEKVQREPLLTLVKEIVPYNMAHNAEHEACDLLMEIEQVDMLEKDIDENAYAKVCLYLTSCVNYVPEPENSALLRCALGVFRKFSRFPEALRLALMLNDMELVEDIFTSCKDVVVQKQMAFMLGRHGVFLELSEDVEEYEDLTEIMSNVQLNSNFLALARELDIMEPKVPDDIYKTHLENNRFGGSGSQVDSARMNLASSFVNGFVNAAFGQDKLLTDDGNKWLYKNKDHGMLSAAASLGMILLWDVDGGLTQIDKYLYSSEDYIKSGALLACGIVNSGVRNECDPALALLSDYVLHNSNTMRLGSIFGLGLAYAGSNREDVLTLLLPVMGDSKSSMEVAGVTALACGMIAVGSCNGDVTSTILQTIMEKSETELKDTYARWLPLGLGLNHLGKGEAIEAILAALEVVSEPFRSFANTLVDVCAYAGSGNVLKVQQLLHICSEHFDSKEKEEDKDKKEKKDKDKKEAPADMGAHQGVAVLGIALIAMGEEIGAEMALRTFGHLLRYGEPTLRRAVPLALALISVSNPRLNILDTLSKFSHDADPEVSYNSIFAMGMVGSGTNNARLAAMLRQLAQYHAKDPNNLFMVRLAQGLTHLGKGTLTLCPYHSDRQLMSQVAVAGLLTVLVSFLDVRNIILGKSHYVLYGLVAAMQPRMLVTFDEELRPLPVSVRVGQAVDVVGQAGKPKTITGFQTHTTPVLLAHGERAELATEEFLPVTPILEGFVILRKNPNYDL",
# PSMD3_HUMAN
"MKQEGSARRRGADKAKPPPGGGEQEPPPPPAPQDVEMKEEAATGGGSTGEADGKTAAAAAEHSQRELDTVTLEDIKEHVKQLEKAVSGKEPRFVLRALRMLPSTSRRLNHYVLYKAVQGFFTSNNATRDFLLPFLEEPMDTEADLQFRPRTGKAASTPLLPEVEAYLQLLVVIFMMNSKRYKEAQKISDDLMQKISTQNRRALDLVAAKCYYYHARVYEFLDKLDVVRSFLHARLRTATLRHDADGQATLLNLLLRNYLHYSLYDQAEKLVSKSVFPEQANNNEWARYLYYTGRIKAIQLEYSEARRTMTNALRKAPQHTAVGFKQTVHKLLIVVELLLGEIPDRLQFRQPSLKRSLMPYFLLTQAVRTGNLAKFNQVLDQFGEKFQADGTYTLIIRLRHNVIKTGVRMISLSYSRISLADIAQKLQLDSPEDAEFIVAKAIRDGVIEASINHEKGYVQSKEMIDIYSTREPQLAFHQRISFCLDIHNMSVKAMRFPPKSYNKDLESAEERREREQQDLEFAKEMAEDDDDSFP",
# PSMD4_HUMAN
"MVLESTMVCVDNSEYMRNGDFLPTRLQAQQDAVNIVCHSKTRSNPENNVGLITLANDCEVLTTLTPDTGRILSKLHTVQPKGKITFCTGIRVAHLALKHRQGKNHKMRIIAFVGSPVEDNEKDLVKLAKRLKKEKVNVDIINFGEEEVNTEKLTAFVNTLNGKDGTGSHLVTVPPGPSLADALISSPILAGEGGAMLGLGASDFEFGVDPSADPELALALRVSMEEQRQRQEEEARRAAAASAAEAGIATTGTEDSDDALLKMTISQQEFGRTGLPDLSSMTEEEQIAYAMQMSLQGAEFGQAESADIDASSAMDTSEPAKEEDDYDVMQDPEFLQSVLENLPGVDPNNEAIRNAMGSLASQATKDGKKDKKEEDKK",
# PSMD5_HUMAN
"MAAQALALLREVARLEAPLEELRALHSVLQAVPLNELRQQAAELRLGPLFSLLNENHREKTTLCVSILERLLQAMEPVHVARNLRVDLQRGLIHPDDSVKILTLSQIGRIVENSDAVTEILNNAELLKQIVYCIGGENLSVAKAAIKSLSRISLTQAGLEALFESNLLDDLKSVMKTNDIVRYRVYELIIEISSVSPESLNYCTTSGLVTQLLRELTGEDVLVRATCIEMVTSLAYTHHGRQYLAQEGVIDQISNIIVGADSDPFSSFYLPGFVKFFGNLAVMDSPQQICERYPIFVEKVFEMIESQDPTMIGVAVDTVGILGSNVEGKQVLQKTGTRFERLLMRIGHQSKNAPVELKIRCLDAISSLLYLPPEQQTDDLLRMTESWFSSLSRDPLELFRGISSQPFPELHCAALKVFTAIANQPWAQKLMFNSPGFVEYVVDRSVEHDKASKDAKYELVKALANSKTIAEIFGNPNYLRLRTYLSEGPYYVKPVSTTAVEGAE",
# PSMD6_HUMAN
"MPLENLEEEGLPKNPDLRIAQLRFLLSLPEHRGDAAVRDELMAAVRDNNMAPYYEALCKSLDWQIDVDLLNKMKKANEDELKRLDEELEDAEKNLGESEIRDAMMAKAEYLCRIGDKEGALTAFRKTYDKTVALGHRLDIVFYLLRIGLFYMDNDLITRNTEKAKSLIEEGGDWDRRNRLKVYQGLYCVAIRDFKQAAELFLDTVSTFTSYELMDYKTFVTYTVYVSMIALERPDLREKVIKGAEILEVLHSLPAVRQYLFSLYECRYSVFFQSLAVVEQEMKKDWLFAPHYRYYVREMRIHAYSQLLESYRSLTLGYMAEAFGVGVEFIDQELSRFIAAGRLHCKIDKVNEIVETNRPDSKNWQYQETIKKGDLLLNRVQKLSRVINM",
# PSMD7_HUMAN
"MPELAVQKVVVHPLVLLSVVDHFNRIGKVGNQKRVVGVLLGSWQKKVLDVSNSFAVPFDEDDKDDSVWFLDHDYLENMYGMFKKVNARERIVGWYHTGPKLHKNDIAINELMKRYCPNSVLVIIDVKPKDLGLPTEAYISVEEVHDDGTPTSKTFEHVTSEIGAEEAEEVGVEHLLRDIKDTTVGTLSQRITNQVHGLKGLNSKLLDIRSYLEKVATGKLPINHQIIYQLQDVFNLLPDVSLQEFVKAFYLKTNDQMVVVYLASLIRSVVALHNLINNKIANRDAEKKEGQEKEESKKDRKEDKEKDKDKEKSDVKKEEKKEKK",
# PSMD8_HUMAN
"MFIKGRAPRAPPRERRRATRGGLRQVVAPPRALGSTSRPHFRRASVCRRRCRKSGGLLAASRKMAAAAVNGAAGFSSSGPAATSGAVLQAATGMYEQLKGEWNRKSPNLSKCGEELGRLKLVLLELNFLPTTGTKLTKQQLILARDILEIGAQWSILRKDIPSFERYMAQLKCYYFDYKEQLPESAYMHQLLGLNLLFLLSQNRVAEFHTELERLPAKDIQTNVYIKHPVSLEQYLMEGSYNKVFLAKGNIPAESYTFFIDILLDTIRDEIAGCIEKAYEKILFTEATRILFFNTPKKMTDYAKKRGWVLGPNNYYSFASQQQKPEDTTIPSTELAKQVIEYARQLEMIV",
# PSMD9_HUMAN
"MSDEEARQSGGSSQAGVVTVSDVQELMRRKEEIEAQIKANYDVLESQKGIGMNEPLVDCEGYPRSDVDLYQVRTARHNIICLQNDHKAVMKQVEEALHQLHARDKEKQARDMAEAHKEAMSRKLGQSESQGPPRAFAKVNSISPGSPASIAGLQVDDEIVEFGSVNTQNFQSLHNIGSVVQHSEGKPLNVTVIRRGEKHQLRLVPTRWAGKGLLGCNIIPLQR",
# PSD10_HUMAN
"MEGCVSNLMVCNLAYSGKLEELKESILADKSLATRTDQDSRTALHWACSAGHTEIVEFLLQLGVPVNDKDDAGWSPLHIAASAGRDEIVKALLGKGAQVNAVNQNGCTPLHYAASKNRHEIAVMLLEGGANPDAKDHYEATAMHRAAAKGNLKMIHILLYYKASTNIQDTEGNTPLHLACDEERVEEAKLLVSQGASIYIENKEEKTPLQVAKGGLGLILKRMVEG",
# PSD11_HUMAN
"MAAAAVVEFQRAQSLLSTDREASIDILHSIVKRDIQENDEEAVQVKEQSILELGSLLAKTGQAAELGGLLKYVRPFLNSISKAKAARLVRSLLDLFLDMEAATGQEVELCLECIEWAKSEKRTFLRQALEARLVSLYFDTKRYQEALHLGSQLLRELKKMDDKALLVEVQLLESKTYHALSNLPKARAALTSARTTANAIYCPPKLQATLDMQSGIIHAAEEKDWKTAYSYFYEAFEGYDSIDSPKAITSLKYMLLCKIMLNTPEDVQALVSGKLALRYAGRQTEALKCVAQASKNRSLADFEKALTDYRAELRDDPIISTHLAKLYDNLLEQNLIRVIEPFSRVQIEHISSLIKLSKADVERKLSQMILDKKFHGILDQGEGVLIIFDEPPVDKTYEAALETIQNMSKVVDSLYNKAKKLT",
# PSD12_HUMAN
"MADGGSERADGRIVKMEVDYSATVDQRLPECAKLAKEGRLQEVIETLLSLEKQTRTASDMVSTSRILVAVVKMCYEAKEWDLLNENIMLLSKRRSQLKQAVAKMVQQCCTYVEEITDLPIKLRLIDTLRMVTEGKIYVEIERARLTKTLATIKEQNGDVKEAASILQELQVETYGSMEKKERVEFILEQMRLCLAVKDYIRTQIISKKINTKFFQEENTEKLKLKYYNLMIQLDQHEGSYLSICKHYRAIYDTPCIQAESEKWQQALKSVVLYVILAPFDNEQSDLVHRISGDKKLEEIPKYKDLLKLFTTMELMRWSTLVEDYGMELRKGSLESPATDVFGSTEEGEKRWKDLKNRVVEHNIRIMAKYYTRITMKRMAQLLDLSVDESEAFLSNLVVNKTIFAKVDRLAGIINFQRPKDPNNLLNDWSQKLNSLMSLVNKTTHLIAKEEMIHNLQ",
# PSD13_HUMAN
"MKDVPGFLQQSQNSGPGQPAVWHRLEELYTKKLWHQLTLQVLDFVQDPCFAQGDGLIKLYENFISEFEHRVNPLSLVEIILHVVRQMTDPNVALTFLEKTREKVKSSDEAVILCKTAIGALKLNIGDLQVTKETIEDVEEMLNNLPGVTSVHSRFYDLSSKYYQTIGNHASYYKDALRFLGCVDIKDLPVSEQQERAFTLGLAGLLGEGVFNFGELLMHPVLESLRNTDRQWLIDTLYAFNSGNVERFQTLKTAWGQQPDLAANEAQLLRKIQLLCLMEMTFTRPANHRQLTFEEIAKSAKITVNEVELLVMKALSVGLVKGSIDEVDKRVHMTWVQPRVLDLQQIKGMKDRLEFWCTDVKSMEMLVEHQAHDILT",
# PSDE_HUMAN
"MDRLLRLGGGMPGLGQGPPTDAPAVDTAEQVYISSLALLKMLKHGRAGVPMEVMGLMLGEFVDDYTVRVIDVFAMPQSGTGVSVEAVDPVFQAKMLDMLKQTGRPEMVVGWYHSHPGFGCWLSGVDINTQQSFEALSERAVAVVVDPIQSVKGKVVIDAFRLINANMMVLGHEPRQTTSNLGHLNKPSIQALIHGLNRHYYSITINYRKNELEQKMLLNLHKKSWMEGLTLQDYSEHCKHNESVVKEMLELAKNYNKAVEEEDKMTPEQLAIKNVGKQDPKRHLEEHVDVLMTSNIVQCLAAMLDTVVFK",
# SEM1_HUMAN
"MSEKKQPVDLGLLEEDDEFEEFPAEDWAGLDEDEDAHVWEDNWDDDNVEDDFSNQLRAELEKHGYKMETS",
# ADRM1_HUMAN
"MTTSGALFPSLVPGSRGASNKYLVEFRAGKMSLKGTTVTPDKRKGLVYIQQTDDSLIHFCWKDRTSGNVEDDLIIFPDDCEFKRVPQCPSGRVYVLKFKAGSKRLFFWMQEPKTDQDEEHCRKVNEYLNNPPMPGALGASGSSGHELSALGGEGGLQSLLGNMSHSQLMQLIGPAGLGGLGGLGALTGPGLASLLGSSGPPGSSSSSSSRSQSAAVTPSSTTSSTRATPAPSAPAAASATSPSPAPSSGNGASTAASPTQPIQLSDLQSILATMNVPAGPAGGQQVDLASVLTPEIMAPILANADVQERLLPYLPSGESLPQTADEIQNTLTSPQFQQALGMFSAALASGQLGPLMCQFGLPAEAVEAANKGDVEAFAKAMQNNAKPEQKEGDTKDKKDEEEDMSLD",
# PSME1_HUMAN
"MAMLRVQPEAQAKVDVFREDLCTKTENLLGSYFPKKISELDAFLKEPALNEANLSNLKAPLDIPVPDPVKEKEKEERKKQQEKEDKDEKKKGEDEDKGPPCGPVNCNEKIVVLLQRLKPEIKDVIEQLNLVTTWLQLQIPRIEDGNNFGVAVQEKVFELMTSLHTKLEGFHTQISKYFSERGDAVTKAAKQPHVGDYRQLVHELDEAEYRDIRLMVMEIRNAYAVLYDIILKNFEKLKKPRGETKGMIY",
# PSME2_HUMAN
"MAKPCGVRLSGEARKQVEVFRQNLFQEAEEFLYRFLPQKIIYLNQLLQEDSLNVADLTSLRAPLDIPIPDPPPKDDEMETDKQEKKEVHKCGFLPGNEKVLSLLALVKPEVWTLKEKCILVITWIQHLIPKIEDGNDFGVAIQEKVLERVNAVKTKVEAFQTTISKYFSERGDAVAKASKETHVMDYRALVHERDEAAYGELRAMVLDLRAFYAELYHIISSNLEKIVNPKGEEKPSMY",
# PSME3_HUMAN
"MASLLKVDQEVKLKVDSFRERITSEAEDLVANFFPKKLLELDSFLKEPILNIHDLTQIHSDMNLPVPDPILLTNSHDGLDGPTYKKRRLDECEEAFQGTKVFVMPNGMLKSNQQLVDIIEKVKPEIRLLIEKCNTVKMWVQLLIPRIEDGNNFGVSIQEETVAELRTVESEAASYLDQISRYYITRAKLVSKIAKYPHVEDYRRTVTEIDEKEYISLRLIISELRNQYVTLHDMILKNIEKIKRPRSSNAETLY",
# PSME4_HUMAN
"MEPAERAGVGEPPEPGGRPEPGPRGFVPQKEIVYNKLLPYAERLDAESDLQLAQIKCNLGRAVQLQELWPGGLFWTRKLSTYIRLYGRKFSKEDHVLFIKLLYELVSIPKLEISMMQGFARLLINLLKKKELLSRADLELPWRPLYDMVERILYSKTEHLGLNWFPNSVENILKTLVKSCRPYFPADATAEMLEEWRPLMCPFDVTMQKAITYFEIFLPTSLPPELHHKGFKLWFDELIGLWVSVQNLPQWEGQLVNLFARLATDNIGYIDWDPYVPKIFTRILRSLNLPVGSSQVLVPRFLTNAYDIGHAVIWITAMMGGPSKLVQKHLAGLFNSITSFYHPSNNGRWLNKLMKLLQRLPNSVVRRLHRERYKKPSWLTPVPDSHKLTDQDVTDFVQCIIQPVLLAMFSKTGSLEAAQALQNLALMRPELVIPPVLERTYPALETLTEPHQLTATLSCVIGVARSLVSGGRWFPEGPTHMLPLLMRALPGVDPNDFSKCMITFQFIATFSTLVPLVDCSSVLQERNDLTEVERELCSATAEFEDFVLQFMDRCFGLIESSTLEQTREETETEKMTHLESLVELGLSSTFSTILTQCSKEIFMVALQKVFNFSTSHIFETRVAGRMVADMCRAAVKCCPEESLKLFVPHCCSVITQLTMNDDVLNDEELDKELLWNLQLLSEITRVDGRKLLLYREQLVKILQRTLHLTCKQGYTLSCNLLHHLLRSTTLIYPTEYCSVPGGFDKPPSEYFPIKDWGKPGDLWNLGIQWHVPSSEEVSFAFYLLDSFLQPELVKLQHCGDGKLEMSRDDILQSLTIVHNCLIGSGNLLPPLKGEPVTNLVPSMVSLEETKLYTGLEYDLSRENHREVIATVIRKLLNHILDNSEDDTKSLFLIIKIIGDLLQFQGSHKHEFDSRWKSFNLVKKSMENRLHGKKQHIRALLIDRVMLQHELRTLTVEGCEYKKIHQDMIRDLLRLSTSSYSQVRNKAQQTFFAALGAYNFCCRDIIPLVLEFLRPDRQGVTQQQFKGALYCLLGNHSGVCLANLHDWDCIVQTWPAIVSSGLSQAMSLEKPSIVRLFDDLAEKIHRQYETIGLDFTIPKSCVEIAELLQQSKNPSINQILLSPEKIKEGIKRQQEKNADALRNYENLVDTLLDGVEQRNLPWKFEHIGIGLLSLLLRDDRVLPLRAIRFFVENLNHDAIVVRKMAISAVAGILKQLKRTHKKLTINPCEISGCPKPTQIIAGDRPDNHWLHYDSKTIPRTKKEWESSCFVEKTHWGYYTWPKNMVVYAGVEEQPKLGRSREDMTEAEQIIFDHFSDPKFVEQLITFLSLEDRKGKDKFNPRRFCLFKGIFRNFDDAFLPVLKPHLEHLVADSHESTQRCVAEIIAGLIRGSKHWTFEKVEKLWELLCPLLRTALSNITVETYNDWGACIATSCESRDPRKLHWLFELLLESPLSGEGGSFVDACRLYVLQGGLAQQEWRVPELLHRLLKYLEPKLTQVYKNVRERIGSVLTYIFMIDVSLPNTTPTISPHVPEFTARILEKLKPLMDVDEEIQNHVMEENGIGEEDERTQGIKLLKTILKWLMASAGRSFSTAVTEQLQLLPLFFKIAPVENDNSYDELKRDAKLCLSLMSQGLLYPHQVPLVLQVLKQTARSSSWHARYTVLTYLQTMVFYNLFIFLNNEDAVKDIRWLVISLLEDEQLEVREMAATTLSGLLQCNFLTMDSPMQIHFEQLCKTKLPKKRKRDPGSVGDTIPSAELVKRHAGVLGLGACVLSSPYDVPTWMPQLLMNLSAHLNDPQPIEMTVKKTLSNFRRTHHDNWQEHKQQFTDDQLLVLTDLLVSPCYYA",
# PSMF1_HUMAN
"MAGLEVLFASAAPAITCRQDALVCFLHWEVVTHGYFGLGVGDQPGPNDKKSELLPAGWNNNKDLYVLRYEYKDGSRKLLVKAITVESSMILNVLEYGSQQVADLTLNLDDYIDAEHLGDFHRTYKNSEELRSRIVSGIITPIHEQWEKANVSSPHREFPPATAREVDPLRIPPHHPHTSRQPPWCDPLGPFVVGGEDLDPFGPRRGGMIVDPLRSGFPRALIDPSSGLPNRLPPGAVPPGARFDPFGPIGTSPPGPNPDHLPPPGYDDMYL"
]


# Add protein names corresponding to the sequences
protein_names = [
"PSA1_HUMAN",
"PSA2_HUMAN",
"PSA3_HUMAN",
"PSA4_HUMAN",
"PSA5_HUMAN",
"PSA6_HUMAN",
"PSA7_HUMAN",
"PSMA8_HUMAN",
"PSB1_HUMAN",
"PSB2_HUMAN",
"PSB3_HUMAN",
"PSB4_HUMAN",
"PSB5_HUMAN",
"PSB6_HUMAN",
"PSB7_HUMAN",
"PSB8_HUMAN",
"PSB9_HUMAN",
"PSB10_HUMAN",
"PRS4_HUMAN",
"PRS7_HUMAN",
"PRS6A_HUMAN",
"PRS6B_HUMAN",
"PRS8_HUMAN",
"PRS10_HUMAN",
"PSMD1_HUMAN",
"PSMD2_HUMAN",
"PSMD3_HUMAN",
"PSMD4_HUMAN",
"PSMD5_HUMAN",
"PSMD6_HUMAN",
"PSMD7_HUMAN",
"PSMD8_HUMAN",
"PSMD9_HUMAN",
"PSD10_HUMAN",
"PSD11_HUMAN",
"PSD12_HUMAN",
"PSD13_HUMAN",
"PSDE_HUMAN",
"SEM1_HUMAN",
"ADRM1_HUMAN",
"PSME1_HUMAN",
"PSME2_HUMAN",
"PSME3_HUMAN",
"PSME4_HUMAN",
"PSMF1_HUMAN"

]

In [44]:
#Ribosome
sequences = [
# RS2_HUMAN
"MADDAGAAGGPGGPGGPGMGNRGGFRGGFGSGIRGRGRGRGRGRGRGRGARGGKAEDKEWMPVTKLGRLVKDMKIKSLEEIYLFSLPIKESEIIDFFLGASLKDEVLKIMPVQKQTRAGQRTRFKAFVAIGDYNGHVGLGVKCSKEVATAIRGAIILAKLSIVPVRRGYWGNKIGKPHTVPCKVTGRCGSVLVRLIPAPRGTGIVSAPVPKKLLMMAGIDDCYTSARGCTATLGNFAKATFDAISKTYSYLTPDLWKETVFTKSPYQEFTDHLVKTHTRVSVQRTQAPAVATT",
# RS3_HUMAN
"MAVQISKKRKFVADGIFKAELNEFLTRELAEDGYSGVEVRVTPTRTEIIILATRTQNVLGEKGRRIRELTAVVQKRFGFPEGSVELYAEKVATRGLCAIAQAESLRYKLLGGLAVRRACYGVLRFIMESGAKGCEVVVSGKLRGQRAKSMKFVDGLMIHSGDPVNYYVDTAVRHVLLRQGVLGIKVKIMLPWDPTGKIGPKKPLPDHVSIVEPKDEILPTTPISEQKGGKPEPPAMPQPVPTA",
# RS3A_HUMAN
"MAVGKNKRLTKGGKKGAKKKVVDPFSKKDWYDVKAPAMFNIRNIGKTLVTRTQGTKIASDGLKGRVFEVSLADLQNDEVAFRKFKLITEDVQGKNCLTNFHGMDLTRDKMCSMVKKWQTMIEAHVDVKTTDGYLLRLFCVGFTKKRNNQIRKTSYAQHQQVRQIRKKMMEIMTREVQTNDLKEVVNKLIPDSIGKDIEKACQSIYPLHDVFVRKVKMLKKPKFELGKLMELHGEGSSSGKATGDETGAKVERADGYEPPVQESV",
# RS4X_HUMAN
"MARGPKKHLKRVAAPKHWMLDKLTGVFAPRPSTGPHKLRECLPLIIFLRNRLKYALTGDEVKKICMQRFIKIDGKVRTDITYPAGFMDVISIDKTGENFRLIYDTKGRFAVHRITPEEAKYKLCKVRKIFVGTKGIPHLVTHDARTIRYPDPLIKVNDTIQIDLETGKITDFIKFDTGNLCMVTGGANLGRIGVITNRERHPGSFDVVHVKDANGNSFATRLSNIFVIGKGNKPWISLPRGKGIRLTIAEERDKRLAAKQSSG",
# RS4Y1_HUMAN
"MARGPKKHLKRVAAPKHWMLDKLTGVFAPRPSTGPHKLRECLPLIVFLRNRLKYALTGDEVKKICMQRFIKIDGKVRVDVTYPAGFMDVISIEKTGEHFRLVYDTKGRFAVHRITVEEAKYKLCKVRKITVGVKGIPHLVTHDARTIRYPDPVIKVNDTVQIDLGTGKIINFIKFDTGNLCMVIGGANLGRVGVITNRERHPGSFDVVHVKDANGNSFATRLSNIFVIGNGNKPWISLPRGKGIRLTVAEERDKRLATKQSSG",
# RS5_HUMAN
"MTEWETAAPAVAETPDIKLFGKWSTDDVQINDISLQDYIAVKEKYAKYLPHSAGRYAAKRFRKAQCPIVERLTNSMMMHGRNNGKKLMTVRIVKHAFEIIHLLTGENPLQVLVNAIINSGPREDSTRIGRAGTVRRQAVDVSPLRRVNQAIWLLCTGAREAAFRNIKTIAECLADELINAAKGSSNSYAIKKKDELERVAKSNR",
# RS6_HUMAN
"MKLNISFPATGCQKLIEVDDERKLRTFYEKRMATEVAADALGEEWKGYVVRISGGNDKQGFPMKQGVLTHGRVRLLLSKGHSCYRPRRTGERKRKSVRGCIVDANLSVLNLVIVKKGEKDIPGLTDTTVPRRLGPKRASRIRKLFNLSKEDDVRQYVVRKPLNKEGKKPRTKAPKIQRLVTPRVLQHKRRRIALKKQRTKKNKEEAAEYAKLLAKRMKEAKEKRQEQIAKRRRLSSLRASTSKSESSQK",
# RS7_HUMAN
"MFSSSAKIVKPNGEKPDEFESGISQALLELEMNSDLKAQLRELNITAAKEIEVGGGRKAIIIFVPVPQLKSFQKIQVRLVRELEKKFSGKHVVFIAQRRILPKPTRKSRTKNKQKRPRSRTLTAVHDAILEDLVFPSEIVGKRIRVKLDGSRLIKVHLDKAQQNNVEHKVETFSGVYKKLTGKDVNFEFPEFQL",
# RS8_HUMAN
"MGISRDNWHKRRKTGGKRKPYHKKRKYELGRPAANTKIGPRRIHTVRVRGGNKKYRALRLDVGNFSWGSECCTRKTRIIDVVYNASNNELVRTKTLVKNCIVLIDSTPYRQWYESHYALPLGRKKGAKLTPEEEEILNKKRSKKIQKKYDERKKNAKISSLLEEQFQQGKLLACIASRPGQCGRADGYVLEGKELEFYLRKIKARKGK",
# RS9_HUMAN
"MPVARSWVCRKTYVTPRRPFEKSRLDQELKLIGEYGLRNKREVWRVKFTLAKIRKAARELLTLDEKDPRRLFEGNALLRRLVRIGVLDEGKMKLDYILGLKIEDFLERRLQTQVFKLGLAKSIHHARVLIRQRHIRVRKQVVNIPSFIVRLDSQKHIDFSLRSPYGGGRPGRVKRKNAKKGQGGAGAGDDEEED",
# RS10_HUMAN
"MLMPKKNRIAIYELLFKEGVMVAKKDVHMPKHPELADKNVPNLHVMKAMQSLKSRGYVKEQFAWRHFYWYLTNEGIQYLRDYLHLPPEIVPATLRRSRPETGRPRPKGLEGERPARLTRGEADRDTYRRSAVPPGADKKAEAGAGSATEFQFRGGFGRGRGQPPQ",
# RS11_HUMAN
"MADIQTERAYQKQPTIFQNKKRVLLGETGKEKLPRYYKNIGLGFKTPKEAIEGTYIDKKCPFTGNVSIRGRILSGVVTKMKMQRTIVIRRDYLHYIRKYNRFEKRHKNMSVHLSPCFRDVQIGDIVTVGECRPLSKTVRFNVLKVTKAAGTKKQFQKF",
# RS12_HUMAN
"MAEEGIAAGGVMDVNTALQEVLKTALIHDGLARGIREAAKALDKRQAHLCVLASNCDEPMYVKLVEALCAEHQINLIKVDDNKKLGEWVGLCKIDREGKPRKVVGCSCVVVKDYGKESQAKDVIEEYFKCKK",
# RS13_HUMAN
"MGRMHAPGKGLSQSALPYRRSVPTWLKLTSDDVKEQIYKLAKKGLTPSQIGVILRDSHGVAQVRFVTGNKILRILKSKGLAPDLPEDLYHLIKKAVAVRKHLERNRKDKDAKFRLILIESRIHRLARYYKTKRVLPPNWKYESSTASALVA",
# RS14_HUMAN
"MAPRKGKEKKEEQVISLGPQVAEGENVFGVCHIFASFNDTFVHVTDLSGKETICRVTGGMKVKADRDESSPYAAMLAAQDVAQRCKELGITALHIKLRATGGNRTKTPGPGAQSALRALARSGMKIGRIEDVTPIPSDSTRRKGGRRGRRL",
# RS15_HUMAN
"MAEVEQKKKRTFRKFTYRGVDLDQLLDMSYEQLMQLYSARQRRRLNRGLRRKQHSLLKRLRKAKKEAPPMEKPEVVKTHLRDMIILPEMVGSMVGVYNGKTFNQVEIKPEMIGHYLGEFSITYKPVKHGRPGIGATHSSRFIPLK",
# RS15A_HUMAN
"MVRMNVLADALKSINNAEKRGKRQVLIRPCSKVIVRFLTVMMKHGYIGEFEIIDDHRAGKIVVNLTGRLNKCGVISPRFDVQLKDLEKWQNNLLPSRQFGFIVLTTSAGIMDHEEARRKHTGGKILGFFF",
# RS16_HUMAN
"MPSKGPLQSVQVFGRKKTATAVAHCKRGNGLIKVNGRPLEMIEPRTLQYKLLEPVLLLGKERFAGVDIRVRVKGGGHVAQIYAIRQSISKALVAYYQKYVDEASKKEIKDILIQYDRTLLVADPRRCESKKFGGPGARARYQKSYR",
# RS17_HUMAN
"MGRVRTKTVKKAARVIIEKYYTRLGNDFHTNKRVCEEIAIIPSKKLRNKIAGYVTHLMKRIQRGPVRGISIKLQEEERERRDNYVPEVSALDQEIIEVDPDTKEMLKLLDFGSLSNLQVTQPTVGMNFKTPRGPV",
# RS18_HUMAN
"MSLVIPEKFQHILRVLNTNIDGRRKIAFAITAIKGVGRRYAHVVLRKADIDLTKRAGELTEDEVERVITIMQNPRQYKIPDWFLNRQKDVKDGKYSQVLANGLDNKLREDLERLKKIRAHRGLRHFWGLRVRGQHTKTTGRRGRTVGVSKKK",
# RS19_HUMAN
"MPGVTVKDVNQQEFVRALAAFLKKSGKLKVPEWVDTVKLAKHKELAPYDENWFYTRAASTARHLYLRGGAGVGSMTKIYGGRQRNGVMPSHFSRGSKSVARRVLQALEGLKMVEKDQDGGRKLTPQGQRDLDRIAGQVAAANKKH",
# RS20_HUMAN
"MAFKDTGKTPVEPEVAIHRIRITLTSRNVKSLEKVCADLIRGAKEKNLKVKGPVRMPTKTLRITTRKTPCGEGSKTWDRFQMRIHKRLIDLHSPSEIVKQITSISIEPGVEVEVTIADA",
# RS21_HUMAN
"MQNDAGEFVDLYVPRKCSASNRIIGAKDHASIQMNVAEVDKVTGRFNGQFKTYAICGAIRRMGESDDSILRLAKADGIVSKNF",
# RS23_HUMAN
"MGKCRGLRTARKLRSHRRDQKWHDKQYKKAHLGTALKANPFGGASHAKGIVLEKVGVEAKQPNSAIRKCVRVQLIKNGKKITAFVPNDGCLNFIEENDEVLVAGFGRKGHAVGDIPGVRFKVVKVANVSLLALYKGKKERPRS",
# RS24_HUMAN
"MNDTVTIRTRKFMTNRLLQRKQMVIDVLHPGKATVPKTEIREKLAKMYKTTPDVIFVFGFRTHFGGGKTTGFGMIYDSLDYAKKNEPKHRLARHGLYEKKKTSRKQRKERKNRMKKVRGTAKANVGAGKKPKE",
# RS25_HUMAN
"MPPKDDKKKKDAGKSAKKDKDPVNKSGGKAKKKKWSKGKVRDKLNNLVLFDKATYDKLCKEVPNYKLITPAVVSERLKIRGSLARAALQELLSKGLIKLVSKHRAQVIYTRNTKGGDAPAAGEDA",
# RS26_HUMAN
"MTKKRRNNGRAKKGRGHVQPIRCTNCARCVPKDKAIKKFVIRNIVEAAAVRDISEASVFDAYVLPKLYVKLHYCVSCAIHSKVVRNRSREARKDRTPPPRFRPAGAAPRPPPKPM",
# RS27_HUMAN
"MPLAKDLLHPSPEEEKRKHKKKRLVQSPNSYFMDVKCPGCYKITTVFSHAQTVVLCVGCSTVLCQPTGGKARLTEGCSFRRKQH",
# RS27A_HUMAN
"MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGAKKRKKKSYTTPKKNKHKRKKVKLAVLKYYKVDENGKISRLRRECPSDECGAGVFMASHFDRHYCGKCCLTYCFNKPEDK",
# RS28_HUMAN
"MDTSRVQPIKLARVTKVLGRTGSQGQCTQVRVEFMDDTSRSIIRNVKGPVREGDVLTLLESEREARRLR",
# RS29_HUMAN
"MGHQQLYWSHPRKFGQGSRSCRVCSNRHGLIRKYGLNMCRQCFRQYAKDIGFIKLD",
# RL3_HUMAN
"MSHRKFSAPRHGSLGFLPRKRSSRHRGKVKSFPKDDPSKPVHLTAFLGYKAGMTHIVREVDRPGSKVNKKEVVEAVTIVETPPMVVVGIVGYVETPRGLRTFKTVFAEHISDECKRRFYKNWHKSKKKAFTKYCKKWQDEDGKKQLEKDFSSMKKYCQVIRVIAHTQMRLLPLRQKKAHLMEIQVNGGTVAEKLDWARERLEQQVPVNQVFGQDEMIDVIGVTKGKGYKGVTSRWHTKKLPRKTHRGLRKVACIGAWHPARVAFSVARAGQKGYHHRTEINKKIYKIGQGYLIKDGKLIKNNASTDYDLSDKSINPLGGFVHYGEVTNDFVMLKGCVVGTKKRVLTLRKSLLVQTKRRALEKIDLKFIDTTSKFGHGRFQTMEEKKAFMGPLKKDRIAKEEGA",
# RL4_HUMAN
"MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRKNNRQPYAVSELAGHQTSAESWGTGRAVARIPRVRGGGTHRSGQGAFGNMCRGGRMFAPTKTWRRWHRRVNTTQKRYAICSALAASALPALVMSKGHRIEEVPELPLVVEDKVEGYKKTKEAVLLLKKLKAWNDIKKVYASQRMRAGKGKMRNRRRIQRRGPCIIYNEDNGIIKAFRNIPGITLLNVSKLNILKLAPGGHVGRFCIWTESAFRKLDELYGTWRKAASLKSNYNLPMHKMINTDLSRILKSPEIQRALRAPRKKIHRRVLKKNPLKNLRIMLKLNPYAKTMRRNTILRQARNHKLRVDKAAAAAAALQAKSDEKAAVAGKKPVVGKKGKKAAVGVKKQKKPLVGKKAAATKKPAPEKKPAEKKPTTEEKKPAA",
# RL5_HUMAN
"MGFVKVVKNKAYFKRYQVKFRRRREGKTDYYARKRLVIQDKNKYNTPKYRMIVRVTNRDIICQIAYARIEGDMIVCAAYAHELPKYGVKVGLTNYAAAYCTGLLLARRLLNRFGMDKIYEGQVEVTGDEYNVESIDGQPGAFTCYLDAGLARTTTGNKVFGALKGAVDGGLSIPHSTKRFPGYDSESKEFNAEVHRKHIMGQNVADYMRYLMEEDEDAYKKQFSQYIKNSVTPDMMEEMYKKAHAAIRENPVYEKKPKKEVKKKRWNRPKMSLAQKKDRVAQKKASFLRAQERAAES",
# RL6_HUMAN
"MAGEKVEKPDTKEKKPEAKKVDAGGKVKKGNLKAKKPKKGKPHCSRNPVLVRGIGRYSRSAMYSRKAMYKRKYSAAKSKVEKKKKEKVLATVTKPVGGDKNGGTRVVKLRKMPRYYPTEDVPRKLLSHGKKPFSQHVRKLRASITPGTILIILTGRHRGKRVVFLKQLASGLLLVTGPLVLNRVPLRRTHQKFVIATSTKIDISNVKIPKHLTDAYFKKKKLRKPRHQEGEIFDTEKEKYEITEQRKIDQKAVDSQILPKIKAIPQLQGYLRSVFALTNGIYPHKLVF",
# RL7_HUMAN
"MEGVEEKKKEVPAVPETLKKKRRNFAELKIKRLRKKFAQKMLRKARRKLIYEKAKHYHKEYRQMYRTEIRMARMARKAGNFYVPAEPKLAFVIRIRGINGVSPKVRKVLQLLRLRQIFNGTFVKLNKASINMLRIVEPYIAWGYPNLKSVNELIYKRGYGKINKKRIALTDNALIARSLGKYGIICMEDLIHEIYTVGKRFKEANNFLWPFKLSSPRGGMKKKTTHFVEGGDAGNREDQINRLIRRMN",
# RL7A_HUMAN
"MPKGKKAKGKKVAPAPAVVKKQEAKKVVNPLFEKRPKNFGIGQDIQPKRDLTRFVKWPRYIRLQRQRAILYKRLKVPPAINQFTQALDRQTATQLLKLAHKYRPETKQEKKQRLLARAEKKAAGKGDVPTKRPPVLRAGVNTVTTLVENKKAQLVVIAHDVDPIELVVFLPALCRKMGVPYCIIKGKARLGRLVHRKTCTTVAFTQVNSEDKGALAKLVEAIRTNYNDRYDEIRRHWGGNVLGPKSVARIAKLEKAKAKELATKLG",
# RL8_HUMAN
"MGRVIRGQRKGAGSVFRAHVKHRKGAARLRAVDFAERHGYIKGIVKDIIHDPGRGAPLAKVVFRDPYRFKKRTELFIAAEGIHTGQFVYCGKKAQLNIGNVLPVGTMPEGTIVCCLEEKPGDRGKLARASGNYATVISHNPETKKTRVKLPSGSKKVISSANRAVVGVVAGGGRIDKPILKAGRAYHKYKAKRNCWPRVRGVAMNPVEHPFGGGNHQHIGKPSTIRRDAPAGRKVGLIAARRTGRLRGTKTVQEKEN",
# RL9_HUMAN
"MKTILSNQTVDIPENVDITLKGRTVIVKGPRGTLRRDFNHINVELSLLGKKKKRLRVDKWWGNRKELATVRTICSHVQNMIKGVTLGFRYKMRSVYAHFPINVVIQENGSLVEIRNFLGEKYIRRVRMRPGVACSVSQAQKDELILEGNDIELVSNSAALIQQATTVKNKDIRKFLDGIYVSEKGTVQQADE",
# RL10_HUMAN
"MGRRPARCYRYCKNKPYPKSRFCRGVPDAKIRIFDLGRKKAKVDEFPLCGHMVSDEYEQLSSEALEAARICANKYMVKSCGKDGFHIRVRLHPFHVIRINKMLSCAGADRLQTGMRGAFGKPQGTVARVHIGQVIMSIRTKLQNKEHVIEALRRAKFKFPGRQKIHISKKWGFTKFNADEFEDMVAEKRLIPDGCGVKYIPSRGPLDKWRALHS",
# RL10A_HUMAN
"MSSKVSRDTLYEAVREVLHGNQRKRRKFLETVELQISLKNYDPQKDKRFSGTVRLKSTPRPKFSVCVLGDQQHCDEAKAVDIPHMDIEALKKLNKNKKLVKKLAKKYDAFLASESLIKQIPRILGPGLNKAGKFPSLLTHNENMVAKVDEVKSTIKFQMKKVLCLAVAVGHVKMTDDELVYNIHLAVNFLVSLLKKNWQNVRALYIKSTMGKPQRLY",
# RL11_HUMAN
"MAQDQGEKENPMRELRIRKLCLNICVGESGDRLTRAAKVLEQLTGQTPVFSKARYTVRSFGIRRNEKIAVHCTVRGAKAEEILEKGLKVREYELRKNNFSDTGNFGFGIQEHIDLGIKYDPSIGIYGLDFYVVLGRPGFSIADKKRRTGCIGAKHRISKEEAMRWFQQKYDGIILPGK",
# RL12_HUMAN
"MPPKFDPNEIKVVYLRCTGGEVGATSALAPKIGPLGLSPKKVGDDIAKATGDWKGLRITVKLTIQNRQAQIEVVPSASALIIKALKEPPRDRKKQKNIKHSGNITFDEIVNIARQMRHRSLARELSGTIKEILGTAQSVGCNVDGRHPHDIIDDINSGAVECPAS",
# RL13_HUMAN
"MAPSRNGMVLKPHFHKDWQRRVATWFNQPARKIRRRKARQAKARRIAPRPASGPIRPIVRCPTVRYHTKVRAGRGFSLEELRVAGIHKKVARTIGISVDPRRRNKSTESLQANVQRLKEYRSKLILFPRKPSAPKKGDSSAEELKLATQLTGPVMPVRNVYKKEKARVITEEEKNFKAFASLRMARANARLFGIRAKRAKEAAEQDVEKKK",
# RL13A_HUMAN
"MAEVQVLVLDGRGHLLGRLAAIVAKQVLLGRKVVVVRCEGINISGNFYRNKLKYLAFLRKRMNTNPSRGPYHFRAPSRIFWRTVRGMLPHKTKRGQAALDRLKVFDGIPPPYDKKKRMVVPAALKVVRLKPTRKFAYLGRLAHEVGWKYQAVTATLEEKRKEKAKIHYRKKKQLMRLRKQAEKNVEKKIDKYTEVLKTHGLLV",
# RL14_HUMAN
"MVFRRFVEVGRVAYVSFGPHAGKLVAIVDVIDQNRALVDGPCTQVRRQAMPFKCMQLTDFILKFPHSAHQKYVRQAWQKADINTKWAATRWAKKIEARERKAKMTDFDRFKVMKAKKMRNRIIKNEVKKLQKAALLKASPKKAPGTKGTAAAAAAAAAAKVPAKKITAASKKAPAQKVPAQKATGQKAAPAPKAQKGQKAPAQKAPAPKASGKKA",
# RL15_HUMAN
"MGAYKYIQELWRKKQSDVMRFLLRVRCWQYRQLSALHRAPRPTRPDKARRLGYKAKQGYVIYRIRVRRGGRKRPVPKGATYGKPVHHGVNQLKFARSLQSVAEERAGRHCGALRVLNSYWVGEDSTYKFFEVILIDPFHKAIRRNPDTQWITKPVHKHREMRGLTSAGRKSRGLGKGHKFHHTIGGSRRAAWRRRNTLQLHRYR",
# RL17_HUMAN
"MVRYSLDPENPTKSCKSRGSNLRVHFKNTRETAQAIKGMHIRKATKYLKDVTLQKQCVPFRRYNGGVGRCAQAKQWGWTQGRWPKKSAEFLLHMLKNAESNAELKGLDVDSLVIEHIQVNKAPKMRRRTYRAHGRINPYMSSPCHIEMILTEKEQIVPKPEEEVAQKKKISQKKLKKQKLMARE",
# RL18_HUMAN
"MGVDIRHNKDRKVRRKEPKSQDIYLRLLVKLYRFLARRTNSTFNQVVLKRLFMSRTNRPPLSLSRMIRKMKLPGRENKTAVVVGTITDDVRVQEVPKLKVCALRVTSRARSRILRAGGKILTFDQLALDSPKGCGTVLLSGPRKGREVYRHFGKAPGTPHSHTKPYVRSKGRKFERARGRRASRGYKN",
# RL18A_HUMAN
"MKASGTLREYKVVGRCLPTPKCHTPPLYRMRIFAPNHVVAKSRFWYFVSQLKKMKKSSGEIVYCGQVFEKSPLRVKNFGIWLRYDSRSGTHNMYREYRDLTTAGAVTQCYRDMGARHRARAHSIQIMKVEEIAASKCRRPAVKQFHDSKIKFPLPHRVLRRQHKPRFTTKRPNTFF",
# RL19_HUMAN
"MSMLRLQKRLASSVLRCGKKKVWLDPNETNEIANANSRQQIRKLIKDGLIIRKPVTVHSRARCRKNTLARRKGRHMGIGKRKGTANARMPEKVTWMRRMRILRRLLRRYRESKKIDRHMYHSLYLKVKGNVFKNKRILMEHIHKLKADKARKKLLADQAEARRSKTKEARKRREERLQAKKEEIIKTLSKEEETKK",
# RL21_HUMAN
"MTNTKGKRRGTRYMFSRPFRKHGVVPLATYMRIYKKGDIVDIKGMGTVQKGMPHKCYHGKTGRVYNVTQHAVGIVVNKQVKGKILAKRINVRIEHIKHSKSRDSFLKRVKENDQKKKEAKEKGTWVQLKRQPAPPREAHFVRTNGKEPELLEPIPYEFMA",
# RL22_HUMAN
"MAPVKKLVVKGGKKKKQVLKFTLDCTHPVEDGIMDAANFEQFLQERIKVNGKAGNLGGGVVTIERSKSKITVTSEVPFSKRYLKYLTKKYLKKNNLRDWLRVVANSKESYELRYFQINQDEEEEEDED",
# RL23_HUMAN
"MSKRGRGGSSGAKFRISLGLPVGAVINCADNTGAKNLYIISVKGIKGRLNRLPAAGVGDMVMATVKKGKPELRKKVHPAVVIRQRKSYRRKDGVFLYFEDNAGVIVNNKGEMKGSAITGPVAKECADLWPRIASNAGSIA",
# RL23A_HUMAN
"MAPKAKKEAPAPPKAEAKAKALKAKKAVLKGVHSHKKKKIRTSPTFRRPKTLRLRRQPKYPRKSAPRRNKLDHYAIIKFPLTTESAMKKIEDNNTLVFIVDVKANKHQIKQAVKKLYDIDVAKVNTLIRPDGEKKAYVRLAPDYDALDVANKIGII",
# RL24_HUMAN
"MKVELCSFSGYKIYPGHGRRYARTDGKVFQFLNAKCESAFLSKRNPRQINWTVLYRRKHKKGQSEEIQKKRTRRAVKFQRAITGASLADIMAKRNQKPEVRKAQREQAIRAAKEAKKAKQASKKTAMAAAKAPTKAAPKQKIVKPVKVSAPRVGGKR",
# RL26_HUMAN
"MKFNPFVTSDRSKNRKRHFNAPSHIRRKIMSSPLSKELRQKYNVRSMPIRKDDEVQVVRGHYKGQQIGKVVQVYRKKYVIYIERVQREKANGTTVHVGIHPSKVVITRLKLDKDRKKILERKAKSRQVGKEKGKYKEETIEKMQE",
# RL27_HUMAN
"MGKFMKPGKVVLVLAGRYSGRKAVIVKNIDDGTSDRPYSHALVAGIDRYPRKVTAAMGKKKIAKRSKIKSFVKVYNYNHLMPTRYSVDIPLDKTVVNKDVFRDPALKRKARREAKVKFEERYKTGKNKWFFQKLRF",
# RL27A_HUMAN
"MPSRLRKTRKLRGHVSHGHGRIGKHRKHPGGRGNAGGLHHHRINFDKYHPGYFGKVGMKHYHLKRNQSFCPTVNLDKLWTLVSEQTRVNAAKNKTGAAPIIDVVRSGYYKVLGKGKLPKQPVIVKAKFFSRRAEEKIKSVGGACVLVA",
# RL28_HUMAN
"MSAHLQWMVVRNCSSFLIKRNKQTYSTEPNNLKARNSFRYNGLIHRKTVGVEPAADGKGVVVVIKRRSGQRKPATSYVRTTINKNARATLSSIRHMIRKNKYRPDLRMAAIRRASAILRSQKPVMVKRKRTRPTKSS",
# RL29_HUMAN
"MAKSKNHTTHNQSRKWHRNGIKKPRSQRYESLKGVDPKFLRNMRFAKKHNKKGLKKMQANNAKAMSARAEAIKALVKPKEVKPKIPKGVSRKLDRLAYIAHPKLGKRARARIAKGLRLCRPKAKAKAKAKDQTKAQAAAPASVPAQAPKRTQAPTKASE",
# RL30_HUMAN
"MVAAKKTKKSLESINSRLQLVMKSGKYVLGYKQTLKMIRQGKAKLVILANNCPALRKSEIEYYAMLAKTGVHHYSGNNIELGTACGKYYRVCTLAIIDPGDSDIIRSMPEQTGEK",
# RL31_HUMAN
"MAPAKKGGEKKKGRSAINEVVTREYTINIHKRIHGVGFKKRAPRALKEIRKFAMKEMGTPDVRIDTRLNKAVWAKGIRNVPYRIRVRLSRKRNEDEDSPNKLYTLVTYVPVTTFKNLQTVNVDEN",
# RL32_HUMAN
"MAALRPLVKPKIVKKRTKKFIRHQSDRYVKIKRNWRKPRGIDNRVRRRFKGQILMPNIGYGSNKKTKHMLPSGFRKFLVHNVKELEVLLMCNKSYCAEIAHNVSSKNRKAIVERAAQLAIRVTNPNARLRSEENE",
# RL34_HUMAN
"MVQRLTYRRRLSYNTASNKTRLSRTPGNRIVYLYTKKVGKAPKSACGVCPGRLRGVRAVRPKVLMRLSKTKKHVSRAYGGSMCAKCVRDRIKRAFLIEEQKIVVKVLKAQAQSQKAK",
# RL35_HUMAN
"MAKIKARDLRGKKKEELLKQLDDLKVELSQLRVAKVTGGAASKLSKIRVVRKSIARVLTVINQTQKENLRKFYKGKKYKPLDLRPKKTRAMRRRLNKHEENLKTKKQQRKERLYPLRKYAVKA",
# RL35A_HUMAN
"MSGRLWSKAIFAGYKRGLRNQREHTALLKIEGVYARDETEFYLGKRCAYVYKAKNNTVTPGGKPNKTRVIWGKVTRAHGNSGMVRAKFRSNLPAKAIGHRIRVMLYPSRI",
# RL36_HUMAN
"MALRYPMAVGLNKGHKVTKNVSKPRHSRRRGRLTKHTKFVRDMIREVCGFAPYERRAMELLKVSKDKRALKFIKKRVGTHIRAKRKREELSNVLAAMRKAAAKKD",
# RL36A_HUMAN
"MVNVPKTRRTFCKKCGKHQPHKVTQYKKGKDSLYAQGKRRYDRKQSGYGGQTKPIFRKKAKTTKKIVLRLECVEPNCRSKRMLAIKRCKHFELGGDKKRKGQVIQF",
# RL37_HUMAN
"MTKGTSSFGKRRNKTHTLCRRCGSKAYHLQKSTCGKCGYPAKRKRKYNWSAKAKRRNTTGTGRMRHLKIVYRRFRHGFREGTTPKPKRAAVAASSSS",
# RL37A_HUMAN
"MAKRTKKVGIVGKYGTRYGASLRKMVKKIEISQHAKYTCSFCGKTKMKRRAVGIWHCGSCMKTVAGGAWTYNTTSAVTVKSAIRRLKELKDQ",
# RL38_HUMAN
"MPRKIEEIKDFLLTARRKDAKSVKIKKNKDNVKFKVRCSRYLYTLVITDKEKAEKLKQSLPPGLAVKELK",
# RL39_HUMAN
"MSSHKTFRIKRFLAKKQKQNRPIPQWIRMKTGNKIRYNSKRRHWRRTKLGL",
# RS32_HUMAN
"MRAKWRKKRMRRLKRKRRKMRQRSK",
# RLA0_HUMAN
"MPREDRATWKSNYFLKIIQLLDDYPKCFIVGADNVGSKQMQQIRMSLRGKAVVLMGKNTMMRKAIRGHLENNPALEKLLPHIRGNVGFVFTKEDLTEIRDMLLANKVPAAARAGAIAPCEVTVPAQNTGLGPEKTSFFQALGITTKISRGTIEILSDVQLIKTGDKVGASEATLLNMLNISPFSFGLVIQQVFDNGSIYNPEVLDITEETLHSRFLEGVRNVASVCLQIGYPTVASVPHSIINGYKRVLALSVETDYTFPLAEKVKAFLADPSAFVAAAPVAAATTAAPAAAAAPAKVEAKEESEESDEDMGFGLFD",
# RLA1_HUMAN
"MASVSELACIYSALILHDDEVTVTEDKINALIKAAGVNVEPFWPGLFAKALANVNIGSLICNVGAGGPAPAAGAAPAGGPAPSTAAAPAEEKKVEAKKEESEESDDDMGFGLFD",
# RLA2_HUMAN
"MRYVASYLLAALGGNSSPSAKDIKKILDSVGIEADDDRLNKVISELNGKNIEDVIAQGIGKLASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEKKDEKKEESEESDDDMGFGLFD"
]


# Add protein names corresponding to the sequences
protein_names = [
"RS2_HUMAN",
"RS3_HUMAN",
"RS3A_HUMAN",
"RS4X_HUMAN",
"RS4Y1_HUMAN",
"RS5_HUMAN",
"RS6_HUMAN",
"RS7_HUMAN",
"RS8_HUMAN",
"RS9_HUMAN",
"RS10_HUMAN",
"RS11_HUMAN",
"RS12_HUMAN",
"RS13_HUMAN",
"RS14_HUMAN",
"RS15_HUMAN",
"RS15A_HUMAN",
"RS16_HUMAN",
"RS17_HUMAN",
"RS18_HUMAN",
"RS19_HUMAN",
"RS20_HUMAN",
"RS21_HUMAN",
"RS23_HUMAN",
"RS24_HUMAN",
"RS25_HUMAN",
"RS26_HUMAN",
"RS27_HUMAN",
"RS27A_HUMAN",
"RS28_HUMAN",
"RS29_HUMAN",
"RL3_HUMAN",
"RL4_HUMAN",
"RL5_HUMAN",
"RL6_HUMAN",
"RL7_HUMAN",
"RL7A_HUMAN",
"RL8_HUMAN",
"RL9_HUMAN",
"RL10_HUMAN",
"RL10A_HUMAN",
"RL11_HUMAN",
"RL12_HUMAN",
"RL13_HUMAN",
"RL13A_HUMAN",
"RL14_HUMAN",
"RL15_HUMAN",
"RL17_HUMAN",
"RL18_HUMAN",
"RL18A_HUMAN",
"RL19_HUMAN",
"RL21_HUMAN",
"RL22_HUMAN",
"RL23_HUMAN",
"RL23A_HUMAN",
"RL24_HUMAN",
"RL26_HUMAN",
"RL27_HUMAN",
"RL27A_HUMAN",
"RL28_HUMAN",
"RL29_HUMAN",
"RL30_HUMAN",
"RL31_HUMAN",
"RL32_HUMAN",
"RL34_HUMAN",
"RL35_HUMAN",
"RL35A_HUMAN",
"RL36_HUMAN",
"RL36A_HUMAN",
"RL37_HUMAN",
"RL37A_HUMAN",
"RL38_HUMAN",
"RL39_HUMAN",
"RS32_HUMAN",
"RLA0_HUMAN",
"RLA1_HUMAN",
"RLA2_HUMAN"

]

In [49]:
#Synapse
sequences = [
# RPSA2_HUMAN
"MSGALDVLQMKEEDVLKFLAAGTHLGGTNLDFQMEHYIYKRKSDGIYIINLKRTWEKLLLAARAIVAIENPADVSVISSRNTGQRAVLKFAAATGATPIAGRFTPGTFTNQIQAAFWEPRLLVVTDPRADHQPLTEASYVNLPTIALCNTDSPLRYVDIAIPCNNKGAHSVGLMWWMLAREVLRMRGTISREHPWEVMPDLYFYRDPEEIEKEEQAAAEKAVTKEEFQGEWTAPSPEFTATQPEVADWSEGVQVPSVPIQQFPTEDWSAQPATEDWSAAPTAQATEWVGATTDWS",
# GRD2I_HUMAN
"MATTATPATNQGWPEDFGFRLGGSGPCFVLEVAKGSSAHAGGLRPGDQILEVEGLAVGGLSRERLVRLARRCPRVPPSLGVLPAPDGGPGPGSGPAAPTTVLRAPRCGRGLALGRELLRLAGRKRPDAVHRERRRKAQEFSRKVDEILGDQPTAKEQVFAALKQFAAEQRVDDLVWTLTLALPREACGPLLDNLRIFIPKKHRARFDEVVSQGLLGKLCRARRAQGAQRLRRSRSEERPERLLVSTRASAPPRRPDEPPPRRASLLVGGLAGPGGARRTVRVYKGNKSFGFTLRGHGPVWIESVLPGSPADNAALKSGDRILFLNGLDMRNCSHDKVVSMLQGSGAMPTLVVEEGLVPFASDSDSLDSPNPSSALTSLQWVAEILPSSIRVQGRTFSQQLEHLLTPPERYGVCRALESFFQHRNIDTLIVDVYPVLDTPAKQVLWQFIYQLLTYEEQELCQEKIACFLGYTAMTAEPEPELDLESEPTPEPQPRSSLRASSMCRRSLRSQGLEAGLSCGPSECPEMPLPLIPGERQAGDGTSLPETPNPKMMSAVYAELESRLNSSFKGKMGTVSKSRASPPGPSPAVTTGPRTLSGVSWPSERLLPSPCYHPLCSGGLASPSSSESHPYASLDSSRAPSPQPGPGPICPDSPPSPDPTRPPSRRKLFTFSHPVRSRDTDRFLDVLSEQLGPRVTIVDDFLTPENDYEEMSFHDDQGSFVTNERSSASDCISSSEEGSSLTYSSISDHIPPPPLSPPPPPPLPFHDAKPSSRSSDGSRGPAQALAKPLTQLSHPVPPPPPPPLPPPVPCAPPMLSRGLGHRRSETSHMSVKRLRWEQVENSEGTIWGQLGEDSDYDKLSDMVKYLDLELHFGTQKPAKPVPGPEPFRKKEVVEILSHKKAYNTSILLAHLKLSPAELRQVLMSMEPRRLEPAHLAQLLLFAPDADEEQRYQAFREAPGRLSEPDQFVLQMLSVPEYKTRLRSLHFQATLQEKTEEIRGSLECLRQASLELKNSRKLAKILEFVLAMGNYLNDGQPKTNKTTGFKINFLTELNSTKTVDGKSTFLHILAKSLSQHFPELLGFAQDLPTVPLAAKVNQRALTSDLADLHGTISEIQDACQSISPSSEDKFAMVMSSFLETAQPALRALDGLQREAMEELGKALAFFGEDSKATTSEAFFGIFAEFMSKFERALSDLQAGEGLRSSGMVSPLAW",
# PSD1_HUMAN
"MAQGAMRFCSEGDCAISPPRCPRRWLPEGPVPQSPPASMYGSTGSLLRRVAGPGPRGRELGRVTAPCTPLRGPPSPRVAPSPWAPSSPTGQPPPGAQSSVVIFRFVEKASVRPLNGLPAPGGLSRSWDLGGVSPPRPTPALGPGSNRKLRLEASTSDPLPARGGSALPGSRNLVHGPPAPPQVGADGLYSSLPNGLGGPPERLATLFGGPADTGFLNQGDTWSSPREVSSHAQRIARAKWEFFYGSLDPPSSGAKPPEQAPPSPPGVGSRQGSGVAVGRAAKYSETDLDTVPLRCYRETDIDEVLAEREEADSAIESQPSSEGPPGTAYPPAPRPGPLPGPHPSLGSGNEDEDDDEAGGEEDVDDEVFEASEGARPGSRMPLKSPVPFLPGTSPSADGPDSFSCVFEAILESHRAKGTSYTSLASLEALASPGPTQSPFFTFELPPQPPAPRPDPPAPAPLAPLEPDSGTSSAADGPWTQRGEEEEAEARAKLAPGREPPSPCHSEDSLGLGAAPLGSEPPLSQLVSDSDSELDSTERLALGSTDTLSNGQKADLEAAQRLAKRLYRLDGFRKADVARHLGKNNDFSKLVAGEYLKFFVFTGMTLDQALRVFLKELALMGETQERERVLAHFSQRYFQCNPEALSSEDGAHTLTCALMLLNTDLHGHNIGKRMTCGDFIGNLEGLNDGGDFPRELLKALYSSIKNEKLQWAIDEEELRRSLSELADPNPKVIKRISGGSGSGSSPFLDLTPEPGAAVYKHGALVRKVHADPDCRKTPRGKRGWKSFHGILKGMILYLQKEEYKPGKALSETELKNAISIHHALATRASDYSKRPHVFYLRTADWRVFLFQAPSLEQMQSWITRINVVAAMFSAPPFPAAVSSQKKFSRPLLPSAATRLSQEEQVRTHEAKLKAMASELREHRAAQLGKKGRGKEAEEQRQKEAYLEFEKSRYSTYAALLRVKLKAGSEELDAVEAALAQAGSTEDGLPPSHSSPSLQPKPSSQPRAQRHSSEPRPGAGSGRRKP",
# WIPF3_HUMAN
"MPVPPPPPPPLPPPPPPLGAPPPPPPSAPPVSTDTSSLRRADPKGRSALLADIQQGTRLRKVTQINDRSAPQIESSKGTNKEGGGSANTRGASTPPTLGDLFAGGFPVLRPAGQRDVAGGKTGQGPGSRAPSPRLPNKTISGPLIPPASPRLGNTSEAHGAARTAPPRPNVPAPPPPTPPPPPPPLPPPLPSSSPIKTPLVSPPGPLTKGNLPVVAPPVPCAPPPPPPPPPPTPPPLPPASVLSDKAVKPQLAPLHLPPIPPPLPLLPPCGYPGLKAEPASPAQDAQEPPAPPPPLPPYASCSPRASLPAPPLPGVNSSSETPPPLPPKSPSFQAPPQKAGAQALPAPPAPPGSQPFLQKKRHGRPGAGGGKLNPPPAPPARSPTTELSSKSQQATAWTPTQQPGGQLRNGSLHIIDDFESKFTFHSVEDFPPPDEYKPCQKIYPSKIPRSRTPGPWLQAEAVGQSSDDIKGRNSQLSLKTLR",
# SHSA7_HUMAN
"MPALLLLVLLASSAGQARARPSNATSAEPAGPLPALLAHLRRLTGALTGGGGAASPGANGTRTGPAGGAGAAARAPPPAELCHGYYDVMGQYDATFNCSTGSYRFCCGTCHYRFCCEHRHMRLAQASCSNYDTPRWATTPPPLAGGAGGAGGAGGGPGPGQAGWLEGGRTGGAGGRGGEGPGGSTAYVVCGVISFALAVGVGAKVAFSKASRAPRAHRDINVPRALVDILRHQAGPGTRPDRARSSSLTPGIGGPDSMPPRTPKNLYNTVKTPNLDWRALPPPSPSLHYSTLSCSRSFHNLSHLPPSYEAAVKSELNRYSSLKRLAEKDLDEAYLKRRPLELPRGTLPLHALRRPGTGGGYRMEAWGGPEELGLAPAPNPRRVMSQEHLLGDGGRSRYEFTLPRARLVSQEHLLLSSPEALRQSREHLLSPPRSPALPPDPTARASLAASHSNLLLGPGGPPTPLRGLPPPSSLHAHHHHALHGSPQPAWMSDAGGGGGTLARRPPFQRQGTLEQLQFIPGHHLPQHLRTASKNEVTV",
# RHG32_HUMAN
"METESESSTLGDDSVFWLESEVIIQVTDCEEEEREEKFRKMKSSVHSEEDDFVPELHRNVHPRERPDWEETLSAMARGADVPEIPGDLTLKTCGSTASMKVKHVKKLPFTKGHFPKMAECAHFHYENVEFGSIQLSLSEEQNEVMKNGCESKELVYLVQIACQGKSWIVKRSYEDFRVLDKHLHLCIYDRRFSQLSELPRSDTLKDSPESVTQMLMAYLSRLSAIAGNKINCGPALTWMEIDNKGNHLLVHEESSINTPAVGAAHVIKRYTARAPDELTLEVGDIVSVIDMPPKVLSTWWRGKHGFQVGLFPGHCVELINQKVPQSVTNSVPKPVSKKHGKLITFLRTFMKSRPTKQKLKQRGILKERVFGCDLGEHLLNSGFEVPQVLQSCTAFIERYGIVDGIYRLSGVASNIQRLRHEFDSEHVPDLTKEPYVQDIHSVGSLCKLYFRELPNPLLTYQLYEKFSDAVSAATDEERLIKIHDVIQQLPPPHYRTLEFLMRHLSLLADYCSITNMHAKNLAIVWAPNLLRSKQIESACFSGTAAFMEVRIQSVVVEFILNHVDVLFSGRISMAMQEGAASLSRPKSLLVSSPSTKLLTLEEAQARTQAQVNSPIVTENKYIEVGEGPAALQGKFHTIIEFPLERKRPQNKMKKSPVGSWRSFFNLGKSSSVSKRKLQRNESEPSEMKAMALKGGRAEGTLRSAKSEESLTSLHAVDGDSKLFRPRRPRSSSDALSASFNGEMLGNRCNSYDNLPHDNESEEEGGLLHIPALMSPHSAEDVDLSPPDIGVASLDFDPMSFQCSPPKAESECLESGASFLDSPGYSKDKPSANKKDAETGSSQCQTPGSTASSEPVSPLQEKLSPFFTLDLSPTEDKSSKPSSFTEKVVYAFSPKIGRKLSKSPSMSISEPISVTLPPRVSEVIGTVSNTTAQNASSSTWDKCVEERDATNRSPTQIVKMKTNETVAQEAYESEVQPLDQVAAEEVELPGKEDQSVSSSQSKAVASGQTQTGAVTHDPPQDSVPVSSVSLIPPPPPPKNVARMLALALAESAQQASTQSLKRPGTSQAGYTNYGDIAVATTEDNLSSSYSAVALDKAYFQTDRPAEQFHLQNNAPGNCDHPLPETTATGDPTHSNTTESGEQHHQVDLTGNQPHQAYLSGDPEKARITSVPLDSEKSDDHVSFPEDQSGKNSMPTVSFLDQDQSPPRFYSGDQPPSYLGASVDKLHHPLEFADKSPTPPNLPSDKIYPPSGSPEENTSTATMTYMTTTPATAQMSTKEASWDVAEQPTTADFAAATLQRTHRTNRPLPPPPSQRSAEQPPVVGQVQAATNIGLNNSHKVQGVVPVPERPPEPRAMDDPASAFISDSGAAAAQCPMATAVQPGLPEKVRDGARVPLLHLRAESVPAHPCGFPAPLPPTRMMESKMIAAIHSSSADATSSSNYHSFVTASSTSVDDALPLPLPVPQPKHASQKTVYSSFARPDVTTEPFGPDNCLHFNMTPNCQYRPQSVPPHHNKLEQHQVYGARSEPPASMGLRYNTYVAPGRNASGHHSKPCSRVEYVSSLSSSVRNTCYPEDIPPYPTIRRVQSLHAPPSSMIRSVPISRTEVPPDDEPAYCPRPLYQYKPYQSSQARSDYHVTQLQPYFENGRVHYRYSPYSSSSSSYYSPDGALCDVDAYGTVQLRPLHRLPNRDFAFYNPRLQGKSLYSYAGLAPRPRANVTGYFSPNDHNVVSMPPAADVKHTYTSWDLEDMEKYRMQSIRRESRARQKVKGPVMSQYDNMTPAVQDDLGGIYVIHLRSKSDPGKTGLLSVAEGKESRHAAKAISPEGEDRFYRRHPEAEMDRAHHHGGHGSTQPEKPSLPQKQSSLRSRKLPDMGCSLPEHRAHQEASHRQFCESKNGPPYPQGAGQLDYGSKGIPDTSEPVSYHNSGVKYAASGQESLRLNHKEVRLSKEMERPWVRQPSAPEKHSRDCYKEEEHLTQSIVPPPKPERSHSLKLHHTQNVERDPSVLYQYQPHGKRQSSVTVVSQYDNLEDYHSLPQHQRGVFGGGGMGTYVPPGFPHPQSRTYATALGQGAFLPAELSLQHPETQIHAE",
# GBRR3_HUMAN
"MVLAFQLVSFTYIWIILKPNVCAASNIKMTHQRCSSSMKQTCKQETRMKKDDSTKARPQKYEQLLHIEDNDFAMRPGFGGSPVPVGIDVHVESIDSISETNMDFTMTFYLRHYWKDERLSFPSTANKSMTFDHRLTRKIWVPDIFFVHSKRSFIHDTTMENIMLRVHPDGNVLLSLRITVSAMCFMDFSRFPLDTQNCSLELESYAYNEDDLMLYWKHGNKSLNTEEHMSLSQFFIEDFSASSGLAFYSSTGWYNRLFINFVLRRHVFFFVLQTYFPAILMVMLSWVSFWIDRRAVPARVSLGITTVLTMSTIITAVSASMPQVSYLKAVDVYLWVSSLFVFLSVIEYAAVNYLTTVEERKQFKKTGKISRMYNIDAVQAMAFDGCYHDSEIDMDQTSLSLNSEDFMRRKSICSPSTDSSRIKRRKSLGGHVGRIILENNHVIDTYSRILFPIVYILFNLFYWGVYV",
# MYO9A_HUMAN
"MNINDGGRRRFEDNEHTLRIYPGAISEGTIYCPIPARKNSTAAEVIESLINKLHLDKTKCYVLAEVKEFGGEEWILNPTDCPVQRMMLWPRMALENRLSGEDYRFLLREKNLDGSIHYGSLQSWLRVTEERRRMMERGFLPQPQQKDFDDLCSLPDLNEKTLLENLRNRFKHEKIYTYVGSILIVINPFKFLPIYNPKYVKMYDNHQLGKLEPHIYAVADVAYHAMLQRKKNQCIVISGESGSGKTQSTNFLIHHLTALSQKGFASGVEQIILGAGPVLEAFGNAKTAHNNNSSRFGKFIQVNYQETGTVLGAYVEKYLLEKSRLVYQEHNERNYHVFYYLLAGASEDERSAFHLKQPEEYHYLNQITKKPLRQSWDDYCYDSEPDCFTVEGEDLRHDFERLQLAMEMVGFLPKTRRQIFSLLSAILHLGNICYKKKTYRDDSIDICNPEVLPIVSELLEVKEEMLFEALVTRKTVTVGEKLILPYKLAEAVTVRNSMAKSLYSALFDWIVFRINHALLNSKDLEHNTKTLSIGVLDIFGFEDYENNSFEQFCINFANERLQHYFNQHIFKLEQEEYRTEGISWHNIDYIDNTCCINLISKKPTGLLHLLDEESNFPQATNQTLLDKFKHQHEDNSYIEFPAVMEPAFIIKHYAGKVKYGVKDFREKNTDHMRPDIVALLRSSKNAFISGMIGIDPVAVFRWAILRAFFRAMVAFREAGKRNIHRKTGHDDTAPCAILKSMDSFSFLQHPVHQRSLEILQRCKEEKYSITRKNPRTPLSDLQGMNALNEKNQHDTFDIAWNGRTGIRQSRLSSGTSLLDKDGIFANSTSSKLLERAHGILTRNKNFKSKPALPKHLLEVNSLKHLTRLTLQDRITKSLLHLHKKKKPPSISAQFQASLSKLMETLGQAEPYFVKCIRSNAEKLPLRFSDVLVLRQLRYTGMLETVRIRQSGYSSKYSFQDFVSHFHVLLPRNIIPSKFNIQDFFRKINLNPDNYQVGKTMVFLKEQERQHLQDLLHQEVLRRIILLQRWFRVLLCRQHFLHLRQASVIIQRFWRNYLNQKQVRDAAVQKDAFVMASAAALLQASWRAHLERQRYLELRAAAIVIQQKWRDYYRRRHMAAICIQARWKAYRESKRYQEQRKKIILLQSTCRGFRARQRFKALKEQRLRETKPEVGLVNIKGYGSLEIQGSDPSGWEDCSFDNRIKAIEECKSVIESNRISRESSVDCLKESPNKQQERAQSQSGVDLQEDVLVRERPRSLEDLHQKKVGRAKRESRRMRELEQAIFSLELLKVRSLGGISPSEDRRWSTELVPEGLQSPRGTPDSESSQGSLELLSYEESQKSKLESVISDEGDLQFPSPKISSSPKFDSRDNALSASNETSSAEHLKDGTMKEMVVCSSESITCKPQLKDSFISNSLPTFFYIPQQDPLKTNSQLDTSIQRNKLLENEDTAGEALTLDINRETRRYHCSGKDQIVPSLNTESSNPVLKKLEKLNTEKEERQKQLQQQNEKEMMEQIRQQTDILEKERKAFKTIEKPRIGECLVAPSSYQSKQRVERPSSLLSLNTSNKGELNVLGSLSLKDAALAQKDSSSAHLPPKDRPVTVFFERKGSPCQSSTVKELSKTDRMGTQLNVACKLSNNRISKREHFRPTQSYSHNSDDLSREGNARPIFFTPKDNMSIPLVSKEALNSKNPQLHKEDEPAWKPVKLAGPGQRETSQRFSSVDEQAKLHKTMSQGEITKLAVRQKASDSDIRPQRAKMRFWAKGKQGEKKTTRVKPTTQSEVSPLFAGTDVIPAHQFPDELAAYHPTPPLSPELPGSCRKEFKENKEPSPKAKRKRSVKISNVALDSMHWQNDSVQIIASVSDLKSMDEFLLKKVNDLDNEDSKKDTLVDVVFKKALKEFRQNIFSFYSSALAMDDGKSIRYKDLYALFEQILEKTMRLEQRDSLGESPVRVWVNTFKVFLDEYMNEFKTSDCTATKVPKTERKKRRKKETDLVEEHNGHIFKATQYSIPTYCEYCSSLIWIMDRASVCKLCKYACHKKCCLKTTAKCSKKYDPELSSRQFGVELSRLTSEDRTVPLVVEKLINYIEMHGLYTEGIYRKSGSTNKIKELRQGLDTDAESVNLDDYNIHVIASVFKQWLRDLPNPLMTFELYEEFLRAMGLQERKETIRGVYSVIDQLSRTHLNTLERLIFHLVRIALQEDTNRMSANALAIVFAPCILRCPDTTDPLQSVQDISKTTTCVELIVVEQMNKYKARLKDISSLEFAENKAKTRLSLIRRSMGKGRIRRGNYPGPSSPVVVRLPSVSDVSEETLTSEAAMETDITEQQQAAMQQEERVLTEQIENLQKEKEELTFEMLVLEPRASDDETLESEASIGTADSSENLNMESEYAISEKSERSLALSSLKTAGKSEPSSKLRKQLKKQQDSLDVVDSSVSSLCLSNTASSHGTRKLFQIYSKSPFYRAASGNEALGMEGPLGQTKFLEDKPQFISRGTFNPEKGKQKLKNVKNSPQKTKETPEGTVMSGRRKTVDPDCTSNQQLALFGNNEFMV",
# SHSA9_HUMAN
"MRRVLRLLLGCFLTELCARVCRAQERAGHGQLAQLGGVLLLAGGNRSGAASGEASEGAEASDAPPTRAPTPDFCRGYFDVMGQWDPPFNCSSGDFIFCCGTCGFRFCCTFKKRRLNQSTCTNYDTPLWLNTGKPPARKDDPLHDPTKDKTNLIVYIICGVVAVMVLVGIFTKLGLEKAHRPQREHMSRALADVMRPQGHCNTDHMERDLNIVVHVQHYENMDTRTPINNLHATQMNNAVPTSPLLQQMGHPHSYPNLGQISNPYEQQPPGKELNKYASLKAVGSSDGDWAVSTLKSPKADKVNDDFYTKRRHLAELAAKGNLPLHPVRVEDEPRAFSPEHGPAKQNGQKSRTNKMPPHPLAYTSTTNFKGWDPNEQSLRRQAYSNKGKLGTAETGSSDPLGTRPQHYPPPQPYFITNSKTEVTV",
# OSCRI_HUMAN
"MRTEPRPPAPSPPSAAAGARAAHPHHAQVHPAVALQPARGVGGQAALFAAGRAGGDLPLQPQPGGAAARRQPAARAAQAFFPAAELAQAGPERQRDPAVASRGGQLHAAGGAGRVPERYP",
# APRIO_HUMAN
"MEHWGQPIPGAGQPWRQPLPTSGRWWLGAASWWWLGAASWWWLGAAPWWWLGTASWWWLGSRRWHPQSVEQAE",
# KIF2A_HUMAN
"MATANFGKIQIGIYVEIKRSDGRIHQAMVTSLNEDNESVTVEWIENGDTKGKEIDLESIFSLNPDLVPDEEIEPSPETPPPPASSAKVNKIVKNRRTVASIKNDPPSRDNRVVGSARARPSQFPEQSSSAQQNGSVSDISPVQAAKKEFGPPSRRKSNCVKEVEKLQEKREKRRLQQQELREKRAQDVDATNPNYEIMCMIRDFRGSLDYRPLTTADPIDEHRICVCVRKRPLNKKETQMKDLDVITIPSKDVVMVHEPKQKVDLTRYLENQTFRFDYAFDDSAPNEMVYRFTARPLVETIFERGMATCFAYGQTGSGKTHTMGGDFSGKNQDCSKGIYALAARDVFLMLKKPNYKKLELQVYATFFEIYSGKVFDLLNRKTKLRVLEDGKQQVQVVGLQEREVKCVEDVLKLIDIGNSCRTSGQTSANAHSSRSHAVFQIILRRKGKLHGKFSLIDLAGNERGADTSSADRQTRLEGAEINKSLLALKECIRALGRNKPHTPFRASKLTQVLRDSFIGENSRTCMIATISPGMASCENTLNTLRYANRVKELTVDPTAAGDVRPIMHHPPNQIDDLETQWGVGSSPQRDDLKLLCEQNEEEVSPQLFTFHEAVSQMVEMEEQVVEDHRAVFQESIRWLEDEKALLEMTEEVDYDVDSYATQLEAILEQKIDILTELRDKVKSFRAALQEEEQASKQINPKRPRAL",
# FZD9_HUMAN
"MAVAPLRGALLLWQLLAAGGAALEIGRFDPERGRGAAPCQAVEIPMCRGIGYNLTRMPNLLGHTSQGEAAAELAEFAPLVQYGCHSHLRFFLCSLYAPMCTDQVSTPIPACRPMCEQARLRCAPIMEQFNFGWPDSLDCARLPTRNDPHALCMEAPENATAGPAEPHKGLGMLPVAPRPARPPGDLGPGAGGSGTCENPEKFQYVEKSRSCAPRCGPGVEVFWSRRDKDFALVWMAVWSALCFFSTAFTVLTFLLEPHRFQYPERPIIFLSMCYNVYSLAFLIRAVAGAQSVACDQEAGALYVIQEGLENTGCTLVFLLLYYFGMASSLWWVVLTLTWFLAAGKKWGHEAIEAHGSYFHMAAWGLPALKTIVILTLRKVAGDELTGLCYVASTDAAALTGFVLVPLSGYLVLGSSFLLTGFVALFHIRKIMKTGGTNTEKLEKLMVKIGVFSILYTVPATCVIVCYVYERLNMDFWRLRATEQPCAAAAGPGGRRDCSLPGGSVPTVAVFMLKIFMSLVVGITSGVWVWSSKTFQTWQSLCYRKIAAGRARAKACRAPGSYGRGTHCHYKAPTVVLHMTKTDPSLENPTHL",
# SNP23_HUMAN
"MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLDEQKEQLNRIEEGLDQINKDMRETEKTLTELNKCCGLCVCPCNRTKNFESGKAYKTTWGDGGENSPCNVVSKQPGPVTNGQLQQPTTGAASGGYIKRITNDAREDEMEENLTQVGSILGNLKDMALNIGNEIDAQNPQIKRITDKADTNRDRIDIANARAKKLIDS",
# KCNK1_HUMAN
"MLQSLAGSSCVRLVERHRSAWCFGFLVLGYLLYLVFGAVVFSSVELPYEDLLRQELRKLKRRFLEEHECLSEQQLEQFLGRVLEASNYGVSVLSNASGNWNWDFTSALFFASTVLSTTGYGHTVPLSDGGKAFCIIYSVIGIPFTLLFLTAVVQRITVHVTRRPVLYFHIRWGFSKQVVAIVHAVLLGFVTVSCFFFIPAAVFSVLEDDWNFLESFYFCFISLSTIGLGDYVPGEGYNQKFRELYKIGITCYLLLGLIAMLVVLETFCELHELKKFRKMFYVKKDKDEDQVHIIEHDQLSFSSITDQAAGMKEDQKQNEPFVATQSSACVDGPANH",
# RB27B_HUMAN
"MTDGDYDYLIKLLALGDSGVGKTTFLYRYTDNKFNPKFITTVGIDFREKRVVYNAQGPNGSSGKAFKVHLQLWDTAGQERFRSLTTAFFRDAMGFLLMFDLTSQQSFLNVRNWMSQLQANAYCENPDIVLIGNKADLPDQREVNERQARELADKYGIPYFETSAATGQNVEKAVETLLDLIMKRMEQCVEKTQIPDTVNGGNSGNLDGEKPPEKKCIC",
# APBB1_HUMAN
"MSVPSSLSQSAINANSHGGPALSLPLPLHAAHNQLLNAKLQATAVGPKDLRSAMGEGGGPEPGPANAKWLKEGQNQLRRAATAHRDQNRNVTLTLAEEASQEPEMAPLGPKGLIHLYSELELSAHNAANRGLRGPGLIISTQEQGPDEGEEKAAGEAEEEEEDDDDEEEEEDLSSPPGLPEPLESVEAPPRPQALTDGPREHSKSASLLFGMRNSAASDEDSSWATLSQGSPSYGSPEDTDSFWNPNAFETDSDLPAGWMRVQDTSGTYYWHIPTGTTQWEPPGRASPSQGSSPQEESQLTWTGFAHGEGFEDGEFWKDEPSDEAPMELGLKEPEEGTLTFPAQSLSPEPLPQEEEKLPPRNTNPGIKCFAVRSLGWVEMTEEELAPGRSSVAVNNCIRQLSYHKNNLHDPMSGGWGEGKDLLLQLEDETLKLVEPQSQALLHAQPIISIRVWGVGRDSGRERDFAYVARDKLTQMLKCHVFRCEAPAKNIATSLHEICSKIMAERRNARCLVNGLSLDHSKLVDVPFQVEFPAPKNELVQKFQVYYLGNVPVAKPVGVDVINGALESVLSSSSREQWTPSHVSVAPATLTILHQQTEAVLGECRVRFLSFLAVGRDVHTFAFIMAAGPASFCCHMFWCEPNAASLSEAVQAACMLRYQKCLDARSQASTSCLPAPPAESVARRVGWTVRRGVQSLWGSLKPKRLGAHTP",
# GRM8_HUMAN
"MVCEGKRSASCPCFFLLTAKFYWILTMMQRTHSQEYAHSIRVDGDIILGGLFPVHAKGERGVPCGELKKEKGIHRLEAMLYAIDQINKDPDLLSNITLGVRILDTCSRDTYALEQSLTFVQALIEKDASDVKCANGDPPIFTKPDKISGVIGAAASSVSIMVANILRLFKIPQISYASTAPELSDNTRYDFFSRVVPPDSYQAQAMVDIVTALGWNYVSTLASEGNYGESGVEAFTQISREIGGVCIAQSQKIPREPRPGEFEKIIKRLLETPNARAVIMFANEDDIRRILEAAKKLNQSGHFLWIGSDSWGSKIAPVYQQEEIAEGAVTILPKRASIDGFDRYFRSRTLANNRRNVWFAEFWEENFGCKLGSHGKRNSHIKKCTGLERIARDSSYEQEGKVQFVIDAVYSMAYALHNMHKDLCPGYIGLCPRMSTIDGKELLGYIRAVNFNGSAGTPVTFNENGDAPGRYDIFQYQITNKSTEYKVIGHWTNQLHLKVEDMQWAHREHTHPASVCSLPCKPGERKKTVKGVPCCWHCERCEGYNYQVDELSCELCPLDQRPNMNRTGCQLIPIIKLEWHSPWAVVPVFVAILGIIATTFVIVTFVRYNDTPIVRASGRELSYVLLTGIFLCYSITFLMIAAPDTIICSFRRVFLGLGMCFSYAALLTKTNRIHRIFEQGKKSVTAPKFISPASQLVITFSLISVQLLGVFVWFVVDPPHIIIDYGEQRTLDPEKARGVLKCDISDLSLICSLGYSILLMVTCTVYAIKTRGVPETFNEAKPIGFTMYTTCIIWLAFIPIFFGTAQSAEKMYIQTTTLTVSMSLSASVSLGMLYMPKVYIIIFHPEQNVQKRKRSFKAVVTAATMQSKLIQKGNDRPNGEVKSELCESLETNTSSTKTTYISYSNHSI",
# MEN1_HUMAN
"MGLKAAQKTLFPLRSIDDVVRLFAAELGREEPDLVLLSLVLGFVEHFLAVNRVIPTNVPELTFQPSPAPDPPGGLTYFPVADLSIIAALYARFTAQIRGAVDLSLYPREGGVSSRELVKKVSDVIWNSLSRSYFKDRAHIQSLFSFITGTKLDSSGVAFAVVGACQALGLRDVHLALSEDHAWVVFGPNGEQTAEVTWHGKGNEDRRGQTVNAGVAERSWLYLKGSYMRCDRKMEVAFMVCAINPSIDLHTDSLELLQLQQKLLWLLYDLGHLERYPMALGNLADLEELEPTPGRPDPLTLYHKGIASAKTYYRDEHIYPYMYLAGYHCRNRNVREALQAWADTATVIQDYNYCREDEEIYKEFFEVANDVIPNLLKEAASLLEAGEERPGEQSQGTQSQGSALQDPECFAHLLRFYDGICKWEEGSPTPVLHVGWATFLVQSLGRFEGQVRQKVRIVSREAEAAEAEEPWGEEAREGRRRGPRRESKPEEPPPPKKPALDKGLGTGQGAVSGPPRKPPGTVAGTARGPEGGSTAQVPAPTASPPPEGPVLTFQSEKMKGMKELLVATKINSSAIKLQLTAQSQVQMKKQKVSTPSDYTLSFLKRQRKGL",
# HIP1_HUMAN
"MDRMASSMKQVPNPLPKVLSRRGVGAGLEAAERESFERTQTVSINKAINTQEVAVKEKHARTCILGTHHEKGAQTFWSVVNRLPLSSNAVLCWKFCHVFHKLLRDGHPNVLKDSLRYRNELSDMSRMWGHLSEGYGQLCSIYLKLLRTKMEYHTKNPRFPGNLQMSDRQLDEAGESDVNNFFQLTVEMFDYLECELNLFQTVFNSLDMSRSVSVTAAGQCRLAPLIQVILDCSHLYDYTVKLLFKLHSCLPADTLQGHRDRFMEQFTKLKDLFYRSSNLQYFKRLIQIPQLPENPPNFLRASALSEHISPVVVIPAEASSPDSEPVLEKDDLMDMDASQQNLFDNKFDDIFGSSFSSDPFNFNSQNGVNKDEKDHLIERLYREISGLKAQLENMKTESQRVVLQLKGHVSELEADLAEQQHLRQQAADDCEFLRAELDELRRQREDTEKAQRSLSEIERKAQANEQRYSKLKEKYSELVQNHADLLRKNAEVTKQVSMARQAQVDLEREKKELEDSLERISDQGQRKTQEQLEVLESLKQELATSQRELQVLQGSLETSAQSEANWAAEFAELEKERDSLVSGAAHREEELSALRKELQDTQLKLASTEESMCQLAKDQRKMLLVGSRKAAEQVIQDALNQLEEPPLISCAGSADHLLSTVTSISSCIEQLEKSWSQYLACPEDISGLLHSITLLAHLTSDAIAHGATTCLRAPPEPADSLTEACKQYGRETLAYLASLEEEGSLENADSTAMRNCLSKIKAIGEELLPRGLDIKQEELGDLVDKEMAATSAAIETATARIEEMLSKSRAGDTGVKLEVNERILGCCTSLMQAIQVLIVASKDLQREIVESGRGTASPKEFYAKNSRWTEGLISASKAVGWGATVMVDAADLVVQGRGKFEELMVCSHEIAASTAQLVAASKVKADKDSPNLAQLQQASRGVNQATAGVVASTISGKSQIEETDNMDFSSMTLTQIKRQEMDSQVRVLELENELQKERQKLGELRKKHYELAGVAEGWEEGTEASPPTLQEVVTEKE",
# EIF3F_HUMAN
"MATPAVPVSAPPATPTPVPAAAPASVPAPTPAPAAAPVPAAAPASSSDPAAAAAATAAPGQTPASAQAPAQTPAPALPGPALPGPFPGGRVVRLHPVILASIVDSYERRNEGAARVIGTLLGTVDKHSVEVTNCFSVPHNESEDEVAVDMEFAKNMYELHKKVSPNELILGWYATGHDITEHSVLIHEYYSREAPNPIHLTVDTSLQNGRMSIKAYVSTLMGVPGRTMGVMFTPLTVKYAYYDTERIGVDLIMKTCFSPNRVIGLSSDLQQVGGASARIQDALSTVLQYAEDVLSGKVSADNTVGRFLMSLVNQVPKIVPDDFETMLNSNINDLLMVTYLANLTQSQIALNEKLVNL",
# CACB4_HUMAN
"MSSSSYAKNGTADGPHSPTSQVARGTTTRRSRLKRSDGSTTSTSFILRQGSADSYTSRPSDSDVSLEEDREAIRQEREQQAAIQLERAKSKPVAFAVKTNVSYCGALDEDVPVPSTAISFDAKDFLHIKEKYNNDWWIGRLVKEGCEIGFIPSPLRLENIRIQQEQKRGRFHGGKSSGNSSSSLGEMVSGTFRATPTSTAKQKQKVTEHIPPYDVVPSMRPVVLVGPSLKGYEVTDMMQKALFDFLKHRFDGRISITRVTADISLAKRSVLNNPSKRAIIERSNTRSSLAEVQSEIERIFELARSLQLVVLDADTINHPAQLIKTSLAPIIVHVKVSSPKVLQRLIKSRGKSQSKHLNVQLVAADKLAQCPPEMFDVILDENQLEDACEHLGEYLEAYWRATHTTSSTPMTPLLGRNLGSTALSPYPTAISGLQSQRMRHSNHSTENSPIERRSLMTSDENYHNERARKSRNRLSSSSQHSRDHYPLVEEDYPDSYQDTYKPHRNRGSPGGYSHDSRHRL",
# EAA5_HUMAN
"MVPHAILARGRDVCRRNGLLILSVLSVIVGCLLGFFLRTRRLSPQEISYFQFPGELLMRMLKMMILPLVVSSLMSGLASLDAKTSSRLGVLTVAYYLWTTFMAVIVGIFMVSIIHPGSAAQKETTEQSGKPIMSSADALLDLIRNMFPANLVEATFKQYRTKTTPVVKSPKVAPEEAPPRRILIYGVQEENGSHVQNFALDLTPPPEVVYKSEPGTSDGMNVLGIVFFSATMGIMLGRMGDSGAPLVSFCQCLNESVMKIVAVAVWYFPFGIVFLIAGKILEMDDPRAVGKKLGFYSVTVVCGLVLHGLFILPLLYFFITKKNPIVFIRGILQALLIALATSSSSATLPITFKCLLENNHIDRRIARFVLPVGATINMDGTALYEAVAAIFIAQVNNYELDFGQIITISITATAASIGAAGIPQAGLVTMVIVLTSVGLPTDDITLIIAVDWALDRFRTMINVLGDALAAGIMAHICRKDFARDTGTEKLLPCETKPVSLQEIVAAQQNGCVKSVAEASELTLGPTCPHHVPVQVEQDEELPAASLNHCTIQISELETNV",
# WASL_HUMAN
"MSSVQQQPPPPRRVTNVGSLLLTPQENESLFTFLGKKCVTMSSAVVQLYAADRNCMWSKKCSGVACLVKDNPQRSYFLRIFDIKDGKLLWEQELYNNFVYNSPRGYFHTFAGDTCQVALNFANEEEAKKFRKAVTDLLGRRQRKSEKRRDPPNGPNLPMATVDIKNPEITTNRFYGPQVNNISHTKEKKKGKAKKKRLTKADIGTPSNFQHIGHVGWDPNTGFDLNNLDPELKNLFDMCGISEAQLKDRETSKVIYDFIEKTGGVEAVKNELRRQAPPPPPPSRGGPPPPPPPPHNSGPPPPPARGRGAPPPPPSRAPTAAPPPPPPSRPSVAVPPPPPNRMYPPPPPALPSSAPSGPPPPPPSVLGVGPVAPPPPPPPPPPPGPPPPPGLPSDGDHQVPTTAGNKAALLDQIREGAQLKKVEQNSRPVSCSGRDALLDQIRQGIQLKSVADGQESTPPTPAPTSGIVGALMEVMQKRSKAIHSSDEDEDEDDEEDFEDDDEWED",
# EF2K_HUMAN
"MADEDLIFRLEGVDGGQSPRAGHDGDSDGDSDDEEGYFICPITDDPSSNQNVNSKVNKYYSNLTKSERYSSSGSPANSFHFKEAWKHAIQKAKHMPDPWAEFHLEDIATERATRHRYNAVTGEWLDDEVLIKMASQPFGRGAMRECFRTKKLSNFLHAQQWKGASNYVAKRYIEPVDRDVYFEDVRLQMEAKLWGEEYNRHKPPKQVDIMQMCIIELKDRPGKPLFHLEHYIEGKYIKYNSNSGFVRDDNIRLTPQAFSHFTFERSGHQLIVVDIQGVGDLYTDPQIHTETGTDFGDGNLGVRGMALFFYSHACNRICESMGLAPFDLSPRERDAVNQNTKLLQSAKTILRGTEEKCGSPQVRTLSGSRPPLLRPLSENSGDENMSDVTFDSLPSSPSSATPHSQKLDHLHWPVFSDLDNMASRDHDHLDNHRESENSGDSGYPSEKRGELDDPEPREHGHSYSNRKYESDEDSLGSSGRVCVEKWNLLNSSRLHLPRASAVALEVQRLNALDLEKKIGKSILGKVHLAMVRYHEGGRFCEKGEEWDQESAVFHLEHAANLGELEAIVGLGLMYSQLPHHILADVSLKETEENKTKGFDYLLKAAEAGDRQSMILVARAFDSGQNLSPDRCQDWLEALHWYNTALEMTDCDEGGEYDGMQDEPRYMMLAREAEMLFTGGYGLEKDPQRSGDLYTQAAEAAMEAMKGRLANQYYQKAEEAWAQMEE",
# DNM1L_HUMAN
"MEALIPVINKLQDVFNTVGADIIQLPQIVVVGTQSSGKSSVLESLVGRDLLPRGTGIVTRRPLILQLVHVSQEDKRKTTGEENGVEAEEWGKFLHTKNKLYTDFDEIRQEIENETERISGNNKGVSPEPIHLKIFSPNVVNLTLVDLPGMTKVPVGDQPKDIELQIRELILRFISNPNSIILAVTAANTDMATSEALKISREVDPDGRRTLAVITKLDLMDAGTDAMDVLMGRVIPVKLGIIGVVNRSQLDINNKKSVTDSIRDEYAFLQKKYPSLANRNGTKYLARTLNRLLMHHIRDCLPELKTRINVLAAQYQSLLNSYGEPVDDKSATLLQLITKFATEYCNTIEGTAKYIETSELCGGARICYIFHETFGRTLESVDPLGGLNTIDILTAIRNATGPRPALFVPEVSFELLVKRQIKRLEEPSLRCVELVHEEMQRIIQHCSNYSTQELLRFPKLHDAIVEVVTCLLRKRLPVTNEMVHNLVAIELAYINTKHPDFADACGLMNNNIEEQRRNRLARELPSAVSRDKSSKVPSALAPASQEPSPAASAEADGKLIQDSRRETKNVASGGGGVGDGVQEPTTGNWRGMLKTSKAEELLAEEKSKPIPIMPASPQKGHAVNLLDVPVPVARKLSAREQRDCEVIERLIKSYFLIVRKNIQDSVPKAVMHFLVNHVKDTLQSELVGQLYKSSLLDDLLTESEDMAQRRKEAADMLKALQGASQIIAEIRETHLW",
# SYT5_HUMAN
"MFPEPPTPGPPSPDTPPDSSRISHGPVPPWALATIVLVSGLLIFSCCFCLYRKSCRRRTGKKSQAQAQVHLQEVKGLGQSYIDKVQPEVEELEPAPSGPGQQVADKHELGRLQYSLDYDFQSGQLLVGILQAMGLAALDLGGSSDPYVRVYLLPDKRRRYETKVHRQTLNPHFGETFAFKVPYVELGGRVLVMAVYDFDRFSRNDAIGEVRVPMSSVDLGRPVQAWRELQAAPREEQEKLGDICFSLRYVPTAGKLTVIVLEAKNLKKMDVGGLSDPYVKVHLLQGGKKVRKKKTTIKKNTLNPYYNEAFSFEVPCDQVQKVQVELTVLDYDKLGKNEAIGRVAVGAAAGGAGLRHWADMLANPRRPIAQWHSLRPPDRVRLLPAP",
# AGRIN_HUMAN
"MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVVLTGTVEEILNVDPVQHTYSCKVRVWRYLKGKDLVARESLLDGGNKVVISGFGDPLICDNQVSTGDTRIFFVNPAPPYLWPAHKNELMLNSSLMRITLRNLEEVEFCVEDKPGTHFTPVPPTPPDACRGMLCGFGAVCEPNAEGPGRASCVCKKSPCPSVVAPVCGSDASTYSNECELQRAQCSQQRRIRLLSRGPCGSRDPCSNVTCSFGSTCARSADGLTASCLCPATCRGAPEGTVCGSDGADYPGECQLLRRACARQENVFKKFDGPCDPCQGALPDPSRSCRVNPRTRRPEMLLRPESCPARQAPVCGDDGVTYENDCVMGRSGAARGLLLQKVRSGQCQGRDQCPEPCRFNAVCLSRRGRPRCSCDRVTCDGAYRPVCAQDGRTYDSDCWRQQAECRQQRAIPSKHQGPCDQAPSPCLGVQCAFGATCAVKNGQAACECLQACSSLYDPVCGSDGVTYGSACELEATACTLGREIQVARKGPCDRCGQCRFGALCEAETGRCVCPSECVALAQPVCGSDGHTYPSECMLHVHACTHQISLHVASAGPCETCGDAVCAFGAVCSAGQCVCPRCEHPPPGPVCGSDGVTYGSACELREAACLQQTQIEEARAGPCEQAECGSGGSGSGEDGDCEQELCRQRGGIWDEDSEDGPCVCDFSCQSVPGSPVCGSDGVTYSTECELKKARCESQRGLYVAAQGACRGPTFAPLPPVAPLHCAQTPYGCCQDNITAARGVGLAGCPSACQCNPHGSYGGTCDPATGQCSCRPGVGGLRCDRCEPGFWNFRGIVTDGRSGCTPCSCDPQGAVRDDCEQMTGLCSCKPGVAGPKCGQCPDGRALGPAGCEADASAPATCAEMRCEFGARCVEESGSAHCVCPMLTCPEANATKVCGSDGVTYGNECQLKTIACRQGLQISIQSLGPCQEAVAPSTHPTSASVTVTTPGLLLSQALPAPPGALPLAPSSTAHSQTTPPPSSRPRTTASVPRTTVWPVLTVPPTAPSPAPSLVASAFGESGSTDGSSDEELSGDQEASGGGSGGLEPLEGSSVATPGPPVERASCYNSALGCCSDGKTPSLDAEGSNCPATKVFQGVLELEGVEGQELFYTPEMADPKSELFGETARSIESTLDDLFRNSDVKKDFRSVRLRDLGPGKSVRAIVDVHFDPTTAFRAPDVARALLRQIQVSRRRSLGVRRPLQEHVRFMDFDWFPAFITGATSGAIAAGATARATTASRLPSSAVTPRAPHPSHTSQPVAKTTAAPTTRRPPTTAPSRVPGRRPPAPQQPPKPCDSQPCFHGGTCQDWALGGGFTCSCPAGRGGAVCEKVLGAPVPAFEGRSFLAFPTLRAYHTLRLALEFRALEPQGLLLYNGNARGKDFLALALLDGRVQLRFDTGSGPAVLTSAVPVEPGQWHRLELSRHWRRGTLSVDGETPVLGESPSGTDGLNLDTDLFVGGVPEDQAAVALERTFVGAGLRGCIRLLDVNNQRLELGIGPGAATRGSGVGECGDHPCLPNPCHGGAPCQNLEAGRFHCQCPPGRVGPTCADEKSPCQPNPCHGAAPCRVLPEGGAQCECPLGREGTFCQTASGQDGSGPFLADFNGFSHLELRGLHTFARDLGEKMALEVVFLARGPSGLLLYNGQKTDGKGDFVSLALRDRRLEFRYDLGKGAAVIRSREPVTLGAWTRVSLERNGRKGALRVGDGPRVLGESPKSRKVPHTVLNLKEPLYVGGAPDFSKLARAAAVSSGFDGAIQLVSLGGRQLLTPEHVLRQVDVTSFAGHPCTRASGHPCLNGASCVPREAAYVCLCPGGFSGPHCEKGLVEKSAGDVDTLAFDGRTFVEYLNAVTESELANEIPVPETLDSGALHSEKALQSNHFELSLRTEATQGLVLWSGKATERADYVALAIVDGHLQLSYNLGSQPVVLRSTVPVNTNRWLRVVAHREQREGSLQVGNEAPVTGSSPLGATQLDTDGALWLGGLPELPVGPALPKAYGTGFVGCLRDVVVGRHPLHLLEDAVTKPELRPCPTP",
# BIN1_HUMAN
"MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQCVQNFNKQLTEGTRLQKDLRTYLASVKAMHEASKKLNECLQEVYEPDWPGRDEANKIAENNDLLWMDYHQKLVDQALLTMDTYLGQFPDIKSRIAKRGRKLVDYDSARHHYESLQTAKKKDEAKIAKPVSLLEKAAPQWCQGKLQAHLVAQTNLLRNQAEEELIKAQKVFEEMNVDLQEELPSLWNSRVGFYVNTFQSIAGLEENFHKEMSKLNQNLNDVLVGLEKQHGSNTFTVKAQPSDNAPAKGNKSPSPPDGSPAATPEIRVNHEPEPAGGATPGATLPKSPSQLRKGPPVPPPPKHTPSKEVKQEQILSLFEDTFVPEISVTTPSQFEAPGPFSEQASLLDLDFDPLPPVTSPVKAPTPSGQSIPWDLWEPTESPAGSLPSGEPSAAEGTFAVSWPSQTAEPGPAQPAEASEVAGGTQPAAGAQEPGETAASEAASSSLPAVVVETFPATVNGTVEGGSGAGRLDLPPGFMFKVQAQHDYTATDTDELQLKAGDVVLVIPFQNPEEQDEGWLMGVKESDWNQHKELEKCRGVFPENFTERVP",
# FAAH1_HUMAN
"MVQYELWAALPGASGVALACCFVAAAVALRWSGRRTARGAVVRARQRQRAGLENMDRAAQRFRLQNPDLDSEALLALPLPQLVQKLHSRELAPEAVLFTYVGKAWEVNKGTNCVTSYLADCETQLSQAPRQGLLYGVPVSLKECFTYKGQDSTLGLSLNEGVPAECDSVVVHVLKLQGAVPFVHTNVPQSMFSYDCSNPLFGQTVNPWKSSKSPGGSSGGEGALIGSGGSPLGLGTDIGGSIRFPSSFCGICGLKPTGNRLSKSGLKGCVYGQEAVRLSVGPMARDVESLALCLRALLCEDMFRLDPTVPPLPFREEVYTSSQPLRVGYYETDNYTMPSPAMRRAVLETKQSLEAAGHTLVPFLPSNIPHALETLSTGGLFSDGGHTFLQNFKGDFVDPCLGDLVSILKLPQWLKGLLAFLVKPLLPRLSAFLSNMKSRSAGKLWELQHEIEVYRKTVIAQWRALDLDVVLTPMLAPALDLNAPGRATGAVSYTMLYNCLDFPAGVVPVTTVTAEDEAQMEHYRGYFGDIWDKMLQKGMKKSVGLPVAVQCVALPWQEELCLRFMREVERLMTPEKQSS",
# CAC1A_HUMAN
"MARFGDEMPARYGGGGSGAAAGVVVGSGGGRGAGGSRQGGQPGAQRMYKQSMAQRARTMALYNPIPVRQNCLTVNRSLFLFSEDNVVRKYAKKITEWPPFEYMILATIIANCIVLALEQHLPDDDKTPMSERLDDTEPYFIGIFCFEAGIKIIALGFAFHKGSYLRNGWNVMDFVVVLTGILATVGTEFDLRTLRAVRVLRPLKLVSGIPSLQVVLKSIMKAMIPLLQIGLLLFFAILIFAIIGLEFYMGKFHTTCFEEGTDDIQGESPAPCGTEEPARTCPNGTKCQPYWEGPNNGITQFDNILFAVLTVFQCITMEGWTDLLYNSNDASGNTWNWLYFIPLIIIGSFFMLNLVLGVLSGEFAKERERVENRRAFLKLRRQQQIERELNGYMEWISKAEEVILAEDETDGEQRHPFDALRRTTIKKSKTDLLNPEEAEDQLADIASVGSPFARASIKSAKLENSTFFHKKERRMRFYIRRMVKTQAFYWTVLSLVALNTLCVAIVHYNQPEWLSDFLYYAEFIFLGLFMSEMFIKMYGLGTRPYFHSSFNCFDCGVIIGSIFEVIWAVIKPGTSFGISVLRALRLLRIFKVTKYWASLRNLVVSLLNSMKSIISLLFLLFLFIVVFALLGMQLFGGQFNFDEGTPPTNFDTFPAAIMTVFQILTGEDWNEVMYDGIKSQGGVQGGMVFSIYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEQEEEEAANQKLALQKAKEVAEVSPLSAANMSIAVKEQQKNQKPAKSVWEQRTSEMRKQNLLASREALYNEMDPDERWKAAYTRHLRPDMKTHLDRPLVVDPQENRNNNTNKSRAAEPTVDQRLGQQRAEDFLRKQARYHDRARDPSGSAGLDARRPWAGSQEAELSREGPYGRESDHHAREGSLEQPGFWEGEAERGKAGDPHRRHVHRQGGSRESRSGSPRTGADGEHRRHRAHRRPGEEGPEDKAERRARHREGSRPARGGEGEGEGPDGGERRRRHRHGAPATYEGDARREDKERRHRRRKENQGSGVPVSGPNLSTTRPIQQDLGRQDPPLAEDIDNMKNNKLATAESAAPHGSLGHAGLPQSPAKMGNSTDPGPMLAIPAMATNPQNAASRRTPNNPGNPSNPGPPKTPENSLIVTNPSGTQTNSAKTARKPDHTTVDIPPACPPPLNHTVVQVNKNANPDPLPKKEEEKKEEEEDDRGEDGPKPMPPYSSMFILSTTNPLRRLCHYILNLRYFEMCILMVIAMSSIALAAEDPVQPNAPRNNVLRYFDYVFTGVFTFEMVIKMIDLGLVLHQGAYFRDLWNILDFIVVSGALVAFAFTGNSKGKDINTIKSLRVLRVLRPLKTIKRLPKLKAVFDCVVNSLKNVFNILIVYMLFMFIFAVVAVQLFKGKFFHCTDESKEFEKDCRGKYLLYEKNEVKARDREWKKYEFHYDNVLWALLTLFTVSTGEGWPQVLKHSVDATFENQGPSPGYRMEMSIFYVVYFVVFPFFFVNIFVALIIITFQEQGDKMMEEYSLEKNERACIDFAISAKPLTRHMPQNKQSFQYRMWQFVVSPPFEYTIMAMIALNTIVLMMKFYGASVAYENALRVFNIVFTSLFSLECVLKVMAFGILNYFRDAWNIFDFVTVLGSITDILVTEFGNNFINLSFLRLFRAARLIKLLRQGYTIRILLWTFVQSFKALPYVCLLIAMLFFIYAIIGMQVFGNIGIDVEDEDSDEDEFQITEHNNFRTFFQALMLLFRSATGEAWHNIMLSCLSGKPCDKNSGILTRECGNEFAYFYFVSFIFLCSFLMLNLFVAVIMDNFEYLTRDSSILGPHHLDEYVRVWAEYDPAAWGRMPYLDMYQMLRHMSPPLGLGKKCPARVAYKRLLRMDLPVADDNTVHFNSTLMALIRTALDIKIAKGGADKQQMDAELRKEMMAIWPNLSQKTLDLLVTPHKSTDLTVGKIYAAMMIMEYYRQSKAKKLQAMREEQDRTPLMFQRMEPPSPTQEGGPGQNALPSTQLDPGGALMAHESGLKESPSWVTQRAQEMFQKTGTWSPEQGPPTDMPNSQPNSQSVEMREMGRDGYSDSEHYLPMEGQGRAASMPRLPAENQRRRGRPRGNNLSTISDTSPMKRSASVLGPKARRLDDYSLERVPPEENQRHHQRRRDRSHRASERSLGRYTDVDTGLGTDLSMTTQSGDLPSKERDQERGRPKDRKHRQHHHHHHHHHHPPPPDKDRYAQERPDHGRARARDQRWSRSPSEGREHMAHRQGSSSVSGSPAPSTSGTSTPRRGRRQLPQTPSTPRPHVSYSPVIRKAGGSGPPQQQQQQQQQQQQQAVARPGRAATSGPRRYPGPTAEPLAGDRPPTGGHSSGRSPRMERRVPGPARSESPRACRHGGARWPASGPHVSEGPPGPRHHGYYRGSDYDEADGPGSGGGEEAMAGAYDAPPPVRHASSGATGRSPRTPRASGPACASPSRHGRRLPNGYYPAHGLARPRGPGSRKGLHEPYSESDDDWC",
# SDCB1_HUMAN
"MSLYPSLEDLKVDKVIQAQTAFSANPANPAILSEASAPIPHDGNLYPRLYPELSQYMGLSLNEEEIRANVAVVSGAPLQGQLVARPSSINYMVAPVTGNDVGIRRAEIKQGIREVILCKDQDGKIGLRLKSIDNGIFVQLVQANSPASLVGLRFGDQVLQINGENCAGWSSDKAHKVLKQAFGEKITMTIRDRPFERTITMHKDSTGHVGFIFKNGKITSIVKDSSAARNGLLTEHNICEINGQNVIGLKDSQIADILSTSGTVVTITIMPAFIFEHIIKRMAPSIMKSLMDHTIPEV",
# DDX3X_HUMAN
"MSHVAVENALGLDQQFAGLDLNSSDNQSGGSTASKGRYIPPHLRNREATKGFYDKDSSGWSSSKDKDAYSSFGSRSDSRGKSSFFSDRGSGSRGRFDDRGRSDYDGIGSRGDRSGFGKFERGGNSRWCDKSDEDDWSKPLPPSERLEQELFSGGNTGINFEKYDDIPVEATGNNCPPHIESFSDVEMGEIIMGNIELTRYTRPTPVQKHAIPIIKEKRDLMACAQTGSGKTAAFLLPILSQIYSDGPGEALRAMKENGRYGRRKQYPISLVLAPTRELAVQIYEEARKFSYRSRVRPCVVYGGADIGQQIRDLERGCHLLVATPGRLVDMMERGKIGLDFCKYLVLDEADRMLDMGFEPQIRRIVEQDTMPPKGVRHTMMFSATFPKEIQMLARDFLDEYIFLAVGRVGSTSENITQKVVWVEESDKRSFLLDLLNATGKDSLTLVFVETKKGADSLEDFLYHEGYACTSIHGDRSQRDREEALHQFRSGKSPILVATAVAARGLDISNVKHVINFDLPSDIEEYVHRIGRTGRVGNLGLATSFFNERNINITKDLLDLLVEAKQEVPSWLENMAYEHHYKGSSRGRSKSSRFSGGFGARDYRQSSGASSSSFSSSRASSSRSGGGGHGSSRGFGGGGYGGFYNSDGYGGNYNSQGVDWWGN",
# PODXL_HUMAN
"MRCALALSALLLLLSTPPLLPSSPSPSPSPSQNATQTTTDSSNKTAPTPASSVTIMATDTAQQSTVPTSKANEILASVKATTLGVSSDSPGTTTLAQQVSGPVNTTVARGGGSGNPTTTIESPKSTKSADTTTVATSTATAKPNTTSSQNGAEDTTNSGGKSSHSVTTDLTSTKAEHLTTPHPTSPLSPRQPTSTHPVATPTSSGHDHLMKISSSSSTVAIPGYTFTSPGMTTTLLETVFHHVSQAGLELLTSGDLPTLASQSAGITASSVISQRTQQTSSQMPASSTAPSSQETVQPTSPATALRTPTLPETMSSSPTAASTTHRYPKTPSPTVAHESNWAKCEDLETQTQSEKQLVLNLTGNTLCAGGASDEKLISLICRAVKATFNPAQDKCGIRLASVPGSQTVVVKEITIHTKLPAKDVYERLKDKWDELKEAGVSDMKLGDQGPPEEAEDRFSMPLIITIVCMASFLLLVAALYGCCHQRLSQRKDQQRLTEELQTVENGYHDNPTLEVMETSSEMQEKKVVSLNGELGDSWIVPLDNLTKDDLDEEEDTHL",
# WNT7A_HUMAN
"MNRKARRCLGHLFLSLGMVYLRIGGFSSVVALGASIICNKIPGLAPRQRAICQSRPDAIIVIGEGSQMGLDECQFQFRNGRWNCSALGERTVFGKELKVGSREAAFTYAIIAAGVAHAITAACTQGNLSDCGCDKEKQGQYHRDEGWKWGGCSADIRYGIGFAKVFVDAREIKQNARTLMNLHNNEAGRKILEENMKLECKCHGVSGSCTTKTCWTTLPQFRELGYVLKDKYNEAVHVEPVRASRNKRPTFLKIKKPLSYRKPMDTDLVYIEKSPNYCEEDPVTGSVGTQGRACNKTAPQASGCDLMCCGRGYNTHQYARVWQCNCKFHWCCYVKCNTCSERTEMYTCK",
# DLGP1_HUMAN
"MKGLSGSRSHHHGVTCDSACDSLSHHSDRKPYLLSPVEHHPADHPYYTQRNSFQAECVGPFSDPLASSTFPRRHYTSQQELKDECALVPRTLATKANRIPANLLDQFERQLPLSRDGYHTLQYKRTAVEHRSDSPGRIRHLVHSVQKLFTKSHSLEGPSKGSVNGGKASPDEAQAARYGKRSKSKERRAEPKARPSTSPGWWSSDDNLDGDMCIYHAPSGVMTMGRCPDRSASQYFLEAYNTISEQAVKASRSNNDVKCSTCANLPVSLDTPLLKKSAWSSTLTVSRAREVYQKASVNMDQAMVKSESCQQERSCQYLQVPQDEWTGYTPRGKDDEIPCRRMRSGSYIKAMGDEDSGDSDTSPKPSPKVAARRESYLKATQPSLTELTTLKISNEHSPKLQIRSHSYLRAVSEVSINRSLDSLDPAGLLTSPKFRSRNESYMRAMSTISQVSEMEVNGQFESVCESVFSELESQAVEALDLPMPGCFRMRSHSYVRAIEKGCSQDDECVSLRSSSPPRTTTTVRTIQSSTVSSCITTYKKTPPPVPPRTTTKPFISITAQSSTESAQDAYMDGQGQRGDIISQSGLSNSTESLDSMKALTAAIEAANAQIHGPASQHMGNNTATVTTTTTIATVTTEDRKKDHFKKNRCLSIGIQVDDAEEPDKTGENKAPSKFQSVGVQVEEEKCFRRFTRSNSVTTAVQADLDFHDNLENSLESIEDNSCPGPMARQFSRDASTSTVSIQGSGNHYHACAADDDFDTDFDPSILPPPDPWIDSITEDPLEAVQRSVCHRDGHWFLKLLQAERDRMEGWCQQMEREERENNLPEDILGKIRTAVGSAQLLMAQKFYQFRELCEENLNPNAHPRPTSQDLAGFWDMLQLSIENISMKFDELHQLKANNWKQMDPLDKKERRAPPPVPKKPAKGPAPLIRERSLESSQRQEARKRLMAAKRAASVRQNSATESAESIEIYIPEAQTRL",
# NRG2_HUMAN
"MRQVCCSALPPPPLEKGRCSSYSDSSSSSSERSSSSSSSSSESGSSSRSSSNNSSISRPAAPPEPRPQQQPQPRSPAARRAAARSRAAAAGGMRRDPAPGFSMLLFGVSLACYSPSLKSVQDQAYKAPVVVEGKVQGLVPAGGSSSNSTREPPASGRVALVKVLDKWPLRSGGLQREQVISVGSCVPLERNQRYIFFLEPTEQPLVFKTAFAPLDTNGKNLKKEVGKILCTDCATRPKLKKMKSQTGQVGEKQSLKCEAAAGNPQPSYRWFKDGKELNRSRDIRIKYGNGRKNSRLQFNKVKVEDAGEYVCEAENILGKDTVRGRLYVNSVSTTLSSWSGHARKCNETAKSYCVNGGVCYYIEGINQLSCKCPNGFFGQRCLEKLPLRLYMPDPKQKAEELYQKRVLTITGICVALLVVGIVCVVAYCKTKKQRKQMHNHLRQNMCPAHQNRSLANGPSHPRLDPEEIQMADYISKNVPATDHVIRRETETTFSGSHSCSPSHHCSTATPTSSHRHESHTWSLERSESLTSDSQSGIMLSSVGTSKCNSPACVEARARRAAAYNLEERRRATAPPYHDSVDSLRDSPHSERYVSALTTPARLSPVDFHYSLATQVPTFEITSPNSAHAVSLPPAAPISYRLAEQQPLLRHPAPPGPGPGPGPGPGPGADMQRSYDSYYYPAAGPGPRRGTCALGGSLGSLPASPFRIPEDDEYETTQECAPPPPPRPRARGASRRTSAGPRRWRRSRLNGLAAQRARAARDSLSLSSGSGGGSASASDDDADDADGALAAESTPFLGLRGAHDALRSDSPPLCPAADSRTYYSLDSHSTRASSRHSRGPPPRAKQDSAPL",
# AGRB1_HUMAN
"MRGQAAAPGPVWILAPLLLLLLLLGRRARAAAGADAGPGPEPCATLVQGKFFGYFSAAAVFPANASRCSWTLRNPDPRRYTLYMKVAKAPVPCSGPGRVRTYQFDSFLESTRTYLGVESFDEVLRLCDPSAPLAFLQASKQFLQMRRQQPPQHDGLRPRAGPPGPTDDFSVEYLVVGNRNPSRAACQMLCRWLDACLAGSRSSHPCGIMQTPCACLGGEAGGPAAGPLAPRGDVCLRDAVAGGPENCLTSLTQDRGGHGATGGWKLWSLWGECTRDCGGGLQTRTRTCLPAPGVEGGGCEGVLEEGRQCNREACGPAGRTSSRSQSLRSTDARRREELGDELQQFGFPAPQTGDPAAEEWSPWSVCSSTCGEGWQTRTRFCVSSSYSTQCSGPLREQRLCNNSAVCPVHGAWDEWSPWSLCSSTCGRGFRDRTRTCRPPQFGGNPCEGPEKQTKFCNIALCPGRAVDGNWNEWSSWSACSASCSQGRQQRTRECNGPSYGGAECQGHWVETRDCFLQQCPVDGKWQAWASWGSCSVTCGAGSQRRERVCSGPFFGGAACQGPQDEYRQCGTQRCPEPHEICDEDNFGAVIWKETPAGEVAAVRCPRNATGLILRRCELDEEGIAYWEPPTYIRCVSIDYRNIQMMTREHLAKAQRGLPGEGVSEVIQTLVEISQDGTSYSGDLLSTIDVLRNMTEIFRRAYYSPTPGDVQNFVQILSNLLAEENRDKWEEAQLAGPNAKELFRLVEDFVDVIGFRMKDLRDAYQVTDNLVLSIHKLPASGATDISFPMKGWRATGDWAKVPEDRVTVSKSVFSTGLTEADEASVFVVGTVLYRNLGSFLALQRNTTVLNSKVISVTVKPPPRSLRTPLEIEFAHMYNGTTNQTCILWDETDVPSSSAPPQLGPWSWRGCRTVPLDALRTRCLCDRLSTFAILAQLSADANMEKATLPSVTLIVGCGVSSLTLLMLVIIYVSVWRYIRSERSVILINFCLSIISSNALILIGQTQTRNKVVCTLVAAFLHFFFLSSFCWVLTEAWQSYMAVTGHLRNRLIRKRFLCLGWGLPALVVAISVGFTKAKGYSTMNYCWLSLEGGLLYAFVGPAAAVVLVNMVIGILVFNKLVSKDGITDKKLKERAGASLWSSCVVLPLLALTWMSAVLAVTDRRSALFQILFAVFDSLEGFVIVMVHCILRREVQDAVKCRVVDRQEEGNGDSGGSFQNGHAQLMTDFEKDVDLACRSVLNKDIAACRTATITGTLKRPSLPEEEKLKLAHAKGPPTNFNSLPANVSKLHLHGSPRYPGGPLPDFPNHSLTLKRDKAPKSSFVGDGDIFKKLDSELSRAQEKALDTSYVILPTATATLRPKPKEEPKYSIHIDQMPQTRLIHLSTAPEASLPARSPPSRQPPSGGPPEAPPAQPPPPPPPPPPPPQQPLPPPPNLEPAPPSLGDPGEPAAHPGPSTGPSTKNENVATLSVSSLERRKSRYAELDFEKIMHTRKRHQDMFQDLNRKLQHAAEKDKEVLGPDSKPEKQQTPNKRPWESLRKAHGTPTWVKKELEPLQPSPLELRSVEWERSGATIPLVGQDIIDLQTEV",
# PTPRT_HUMAN
"MASLAALALSLLLRLQLPPLPGARAQSAAGGCSFDEHYSNCGYSVALGTNGFTWEQINTWEKPMLDQAVPTGSFMMVNSSGRASGQKAHLLLPTLKENDTHCIDFHYYFSSRDRSSPGALNVYVKVNGGPQGNPVWNVSGVVTEGWVKAELAISTFWPHFYQVIFESVSLKGHPGYIAVDEVRVLAHPCRKAPHFLRLQNVEVNVGQNATFQCIAGGKWSQHDKLWLQQWNGRDTALMVTRVVNHRRFSATVSVADTAQRSVSKYRCVIRSDGGSGVSNYAELIVKEPPTPIAPPELLAVGATYLWIKPNANSIIGDGPIILKEVEYRTTTGTWAETHIVDSPNYKLWHLDPDVEYEIRVLLTRPGEGGTGPPGPPLTTRTKCADPVHGPQNVEIVDIRARQLTLQWEPFGYAVTRCHSYNLTVQYQYVFNQQQYEAEEVIQTSSHYTLRGLRPFMTIRLRLLLSNPEGRMESEELVVQTEEDVPGAVPLESIQGGPFEEKIYIQWKPPNETNGVITLYEINYKAVGSLDPSADLSSQRGKVFKLRNETHHLFVGLYPGTTYSFTIKASTAKGFGPPVTTRIATKISAPSMPEYDTDTPLNETDTTITVMLKPAQSRGAPVSVYQLVVKEERLQKSRRAADIIECFSVPVSYRNASSLDSLHYFAAELKPANLPVTQPFTVGDNKTYNGYWNPPLSPLKSYSIYFQALSKANGETKINCVRLATKGASTQNSNTVEPEKQVDNTVKMAGVIAGLLMFIIILLGVMLTIKRRRNAYSYSYYLKLAKKQKETQSGAQREMGPVASADKPTTKLSASRNDEGFSSSSQDVNGFTDGSRGELSQPTLTIQTHPYRTCDPVEMSYPRDQFQPAIRVADLLQHITQMKRGQGYGFKEEYEALPEGQTASWDTAKEDENRNKNRYGNIISYDHSRVRLLVLDGDPHSDYINANYIDGYHRPRHYIATQGPMQETVKDFWRMIWQENSASIVMVTNLVEVGRVKCVRYWPDDTEVYGDIKVTLIETEPLAEYVIRTFTVQKKGYHEIRELRLFHFTSWPDHGVPCYATGLLGFVRQVKFLNPPEAGPIVVHCSAGAGRTGCFIAIDTMLDMAENEGVVDIFNCVRELRAQRVNLVQTEEQYVFVHDAILEACLCGNTAIPVCEFRSLYYNISRLDPQTNSSQIKDEFQTLNIVTPRVRPEDCSIGLLPRNHDKNRSMDVLPLDRCLPFLISVDGESSNYINAALMDSHKQPAAFVVTQHPLPNTVADFWRLVFDYNCSSVVMLNEMDTAQFCMQYWPEKTSGCYGPIQVEFVSADIDEDIIHRIFRICNMARPQDGYRIVQHLQYIGWPAYRDTPPSKRSLLKVVRRLEKWQEQYDGREGRTVVHCLNGGGRSGTFCAICSVCEMIQQQNIIDVFHIVKTLRNNKSNMVETLEQYKFVYEVALEYLSSF",
# FCHO1_HUMAN
"MSYFGEHFWGEKNHGFEVLYHSVKQGPISTKELADFIRERATIEETYSKAMAKLSKLASNGTPMGTFAPLWEVFRVSSDKLALCHLELTRKLQDLIKDVLRYGEEQLKTHKKCKEEVVSTLDAVQVLSGVSQLLPKSRENYLNRCMDQERLRRESTSQKEMDKAETKTKKAAESLRRSVEKYNSARADFEQKMLDSALRFQAMEETHLRHMKALLGSYAHSVEDTHVQIGQVHEEFKQNIENVSVEMLLRKFAESKGTGREKPGPLDFEAYSAAALQEAMKRLRGAKAFRLPGLSRREREPEPPAAVDFLEPDSGTCPEVDEEGFTVRPDVTQNSTAEPSRFSSSDSDFDDEEPRKFYVHIKPAPARAPACSPEAAAAQLRATAGSLILPPGPGGTMKRHSSRDAAGKPQRPRSAPRTSSCAERLQSEEQVSKNLFGPPLESAFDHEDFTGSSSLGFTSSPSPFSSSSPENVEDSGLDSPSHAAPGPSPDSWVPRPGTPQSPPSCRAPPPEARGIRAPPLPDSPQPLASSPGPWGLEALAGGDLMPAPADPTAREGLAAPPRRLRSRKVSCPLTRSNGDLSRSLSPSPLGSSAASTALERPSFLSQTGHGVSRGPSPVVLGSQDALPIATAFTEYVHAYFRGHSPSCLARVTGELTMTFPAGIVRVFSGTPPPPVLSFRLVHTTAIEHFQPNADLLFSDPSQSDPETKDFWLNMAALTEALQRQAEQNPTASYYNVVLLRYQFSRPGPQSVPLQLSAHWQCGATLTQVSVEYGYRPGATAVPTPLTNVQILLPVGEPVTNVRLQPAATWNLEEKRLTWRLPDVSEAGGSGRLSASWEPLSGPSTPSPVAAQFTSEGTTLSGVDLELVGSGYRMSLVKRRFATGMYLVSC",
# RHG33_HUMAN
"MVARSTDSLDGPGEGSVQPLPTAGGPSVKGKPGKRLSAPRGPFPRLADCAHFHYENVDFGHIQLLLSPDREGPSLSGENELVFGVQVTCQGRSWPVLRSYDDFRSLDAHLHRCIFDRRFSCLPELPPPPEGARAAQMLVPLLLQYLETLSGLVDSNLNCGPVLTWMELDNHGRRLLLSEEASLNIPAVAAAHVIKRYTAQAPDELSFEVGDIVSVIDMPPTEDRSWWRGKRGFQVGFFPSECVELFTERPGPGLKADADGPPCGIPAPQGISSLTSAVPRPRGKLAGLLRTFMRSRPSRQRLRQRGILRQRVFGCDLGEHLSNSGQDVPQVLRCCSEFIEAHGVVDGIYRLSGVSSNIQRLRHEFDSERIPELSGPAFLQDIHSVSSLCKLYFRELPNPLLTYQLYGKFSEAMSVPGEEERLVRVHDVIQQLPPPHYRTLEYLLRHLARMARHSANTSMHARNLAIVWAPNLLRSMELESVGMGGAAAFREVRVQSVVVEFLLTHVDVLFSDTFTSAGLDPAGRCLLPRPKSLAGSCPSTRLLTLEEAQARTQGRLGTPTEPTTPKAPASPAERRKGERGEKQRKPGGSSWKTFFALGRGPSVPRKKPLPWLGGTRAPPQPSGSRPDTVTLRSAKSEESLSSQASGAGLQRLHRLRRPHSSSDAFPVGPAPAGSCESLSSSSSSESSSSESSSSSSESSAAGLGALSGSPSHRTSAWLDDGDELDFSPPRCLEGLRGLDFDPLTFRCSSPTPGDPAPPASPAPPAPASAFPPRVTPQAISPRGPTSPASPAALDISEPLAVSVPPAVLELLGAGGAPASATPTPALSPGRSLRPHLIPLLLRGAEAPLTDACQQEMCSKLRGAQGPLGPDMESPLPPPPLSLLRPGGAPPPPPKNPARLMALALAERAQQVAEQQSQQECGGTPPASQSPFHRSLSLEVGGEPLGTSGSGPPPNSLAHPGAWVPGPPPYLPRQQSDGSLLRSQRPMGTSRRGLRGPAQVSAQLRAGGGGRDAPEAAAQSPCSVPSQVPTPGFFSPAPRECLPPFLGVPKPGLYPLGPPSFQPSSPAPVWRSSLGPPAPLDRGENLYYEIGASEGSPYSGPTRSWSPFRSMPPDRLNASYGMLGQSPPLHRSPDFLLSYPPAPSCFPPDHLGYSAPQHPARRPTPPEPLYVNLALGPRGPSPASSSSSSPPAHPRSRSDPGPPVPRLPQKQRAPWGPRTPHRVPGPWGPPEPLLLYRAAPPAYGRGGELHRGSLYRNGGQRGEGAGPPPPYPTPSWSLHSEGQTRSYC",
# NCAN_HUMAN
"MGAPFVWALGLLMLQMLLFVAGEQGTQDITDASERGLHMQKLGSGSVQAALAELVALPCLFTLQPRPSAARDAPRIKWTKVRTASGQRQDLPILVAKDNVVRVAKSWQGRVSLPSYPRRRANATLLLGPLRASDSGLYRCQVVRGIEDEQDLVPLEVTGVVFHYRSARDRYALTFAEAQEACRLSSAIIAAPRHLQAAFEDGFDNCDAGWLSDRTVRYPITQSRPGCYGDRSSLPGVRSYGRRNPQELYDVYCFARELGGEVFYVGPARRLTLAGARAQCRRQGAALASVGQLHLAWHEGLDQCDPGWLADGSVRYPIQTPRRRCGGPAPGVRTVYRFANRTGFPSPAERFDAYCFRAHHPTSQHGDLETPSSGDEGEILSAEGPPVRELEPTLEEEEVVTPDFQEPLVSSGEEETLILEEKQESQQTLSPTPGDPMLASWPTGEVWLSTVAPSPSDMGAGTAASSHTEVAPTDPMPRRRGRFKGLNGRYFQQQEPEPGLQGGMEASAQPPTSEAAVNQMEPPLAMAVTEMLGSGQSRSPWADLTNEVDMPGAGSAGGKSSPEPWLWPPTMVPPSISGHSRAPVLELEKAEGPSARPATPDLFWSPLEATVSAPSPAPWEAFPVATSPDLPMMAMLRGPKEWMLPHPTPISTEANRVEAHGEATATAPPSPAAETKVYSLPLSLTPTGQGGEAMPTTPESPRADFRETGETSPAQVNKAEHSSSSPWPSVNRNVAVGFVPTETATEPTGLRGIPGSESGVFDTAESPTSGLQATVDEVQDPWPSVYSKGLDASSPSAPLGSPGVFLVPKVTPNLEPWVATDEGPTVNPMDSTVTPAPSDASGIWEPGSQVFEEAESTTLSPQVALDTSIVTPLTTLEQGDKVGVPAMSTLGSSSSQPHPEPEDQVETQGTSGASVPPHQSSPLGKPAVPPGTPTAASVGESASVSSGEPTVPWDPSSTLLPVTLGIEDFELEVLAGSPGVESFWEEVASGEEPALPGTPMNAGAEEVHSDPCENNPCLHGGTCNANGTMYGCSCDQGFAGENCEIDIDDCLCSPCENGGTCIDEVNGFVCLCLPSYGGSFCEKDTEGCDRGWHKFQGHCYRYFAHRRAWEDAEKDCRRRSGHLTSVHSPEEHSFINSFGHENTWIGLNDRIVERDFQWTDNTGLQFENWRENQPDNFFAGGEDCVVMVAHESGRWNDVPCNYNLPYVCKKGTVLCGPPPAVENASLIGARKAKYNVHATVRYQCNEGFAQHHVATIRCRSNGKWDRPQIVCTKPRRSHRMRRHHHHHQHHHQHHHHKSRKERRKHKKHPTEDWEKDEGNFC",
# IF1AY_HUMAN
"MPKNKGKGGKNRRRGKNENESEKRELVFKEDGQEYAQVIKMLGNGRLEALCFDGVKRLCHIRGKLRKKVWINTSDIILVGLRDYQDNKADVILKYNADEARSLKAYGELPEHAKINETDTFGPGDDDEIQFDDIGDDDEDIDDI",
# AP3D1_HUMAN
"MALKMVKGSIDRMFDKNLQDLVRGIRNHKEDEAKYISQCIDEIKQELKQDNIAVKANAVCKLTYLQMLGYDISWAAFNIIEVMSASKFTFKRIGYLAASQSFHEGTDVIMLTTNQIRKDLSSPSQYDTGVALTGLSCFVTPDLARDLANDIMTLMSHTKPYIRKKAVLIMYKVFLKYPESLRPAFPRLKEKLEDPDPGVQSAAVNVICELARRNPKNYLSLAPLFFKLMTSSTNNWVLIKIIKLFGALTPLEPRLGKKLIEPLTNLIHSTSAMSLLYECVNTVIAVLISLSSGMPNHSASIQLCVQKLRILIEDSDQNLKYLGLLAMSKILKTHPKSVQSHKDLILQCLDDKDESIRLRALDLLYGMVSKKNLMEIVKKLMTHVDKAEGTTYRDELLTKIIDICSQSNYQYITNFEWYISILVELTRLEGTRHGHLIAAQMLDVAIRVKAIRKFAVSQMSALLDSAHLLASSTQRNGICEVLYAAAWICGEFSEHLQEPHHTLEAMLRPRVTTLPGHIQAVYVQNVVKLYASILQQKEQAGEAEGAQAVTQLMVDRLPQFVQSADLEVQERASCILQLVKHIQKLQAKDVPVAEEVSALFAGELNPVAPKAQKKVPVPEGLDLDAWINEPLSDSESEDERPRAVFHEEEQRRPKHRPSEADEEELARRREARKQEQANNPFYIKSSPSPQKRYQDTPGVEHIPVVQIDLSVPLKVPGLPMSDQYVKLEEERRHRQKLEKDKRRKKRKEKEKKGKRRHSSLPTESDEDIAPAQQVDIVTEEMPENALPSDEDDKDPNDPYRALDIDLDKPLADSEKLPIQKHRNTETSKSPEKDVPMVEKKSKKPKKKEKKHKEKERDKEKKKEKEKKKSPKPKKKKHRKEKEERTKGKKKSKKQPPGSEEAAGEPVQNGAPEEEQLPPESSYSLLAENSYVKMTCDIRGSLQEDSQVTVAIVLENRSSSILKGMELSVLDSLNARMARPQGSSVHDGVPVPFQLPPGVSNEAQYVFTIQSIVMAQKLKGTLSFIAKNDEGATHEKLDFRLHFSCSSYLITTPCYSDAFAKLLESGDLSMSSIKVDGIRMSFQNLLAKICFHHHFSVVERVDSCASMYSRSIQGHHVCLLVKKGENSVSVDGKCSDSTLLSNLLEEMKATLAKC",
# ABLM1_HUMAN
"MPAFLGLKCLGKLCSSEKSKVTSSERTSARGSNRKRLIVEDRRVSGTSFTAHRRATITHLLYLCPKDYCPRGRVCNSVDPFVAHPQDPHHPSEKPVIHCHKCGEPCKGEVLRVQTKHFHIKCFTCKVCGCDLAQGGFFIKNGEYLCTLDYQRMYGTRCHGCGEFVEGEVVTALGKTYHPNCFACTICKRPFPPGDRVTFNGRDCLCQLCAQPMSSSPKETTFSSNCAGCGRDIKNGQALLALDKQWHLGCFKCKSCGKVLTGEYISKDGAPYCEKDYQGLFGVKCEACHQFITGKVLEAGDKHYHPSCARCSRCNQMFTEGEEMYLQGSTVWHPDCKQSTKTEEKLRPTRTSSESIYSRPGSSIPGSPGHTIYAKVDNEILDYKDLAAIPKVKAIYDIERPDLITYEPFYTSGYDDKQERQSLGESPRTLSPTPSAEGYQDVRDRMIHRSTSQGSINSPVYSRHSYTPTTSRSPQHFHRPGNEPSSGRNSPLPYRPDSRPLTPTYAQAPKHFHVPDQGINIYRKPPIYKQHAALAAQSKSSEDIIKFSKFPAAQAPDPSETPKIETDHWPGPPSFAVVGPDMKRRSSGREEDDEELLRRRQLQEEQLMKLNSGLGQLILKEEMEKESRERSSLLASRYDSPINSASHIPSSKTASLPGYGRNGLHRPVSTDFAQYNSYGDVSGGVRDYQTLPDGHMPAMRMDRGVSMPNMLEPKIFPYEMLMVTNRGRNKILREVDRTRLERHLAPEVFREIFGMSIQEFDRLPLWRRNDMKKKAKLF",
# DVL1_HUMAN
"MAETKIIYHMDEEETPYLVKLPVAPERVTLADFKNVLSNRPVHAYKFFFKSMDQDFGVVKEEIFDDNAKLPCFNGRVVSWLVLAEGAHSDAGSQGTDSHTDLPPPLERTGGIGDSRPPSFHPNVASSRDGMDNETGTESMVSHRRERARRRNREEAARTNGHPRGDRRRDVGLPPDSASTALSSELESSSFVDSDEDGSTSRLSSSTEQSTSSRLIRKHKRRRRKQRLRQADRASSFSSITDSTMSLNIVTVTLNMERHHFLGISIVGQSNDRGDGGIYIGSIMKGGAVAADGRIEPGDMLLQVNDVNFENMSNDDAVRVLREIVSQTGPISLTVAKCWDPTPRSYFTVPRADPVRPIDPAAWLSHTAALTGALPRYGTSPCSSAVTRTSSSSLTSSVPGAPQLEEAPLTVKSDMSAVVRVMQLPDSGLEIRDRMWLKITIANAVIGADVVDWLYTHVEGFKERREARKYASSLLKHGFLRHTVNKITFSEQCYYVFGDLCSNLATLNLNSGSSGTSDQDTLAPLPHPAAPWPLGQGYPYQYPGPPPCFPPAYQDPGFSYGSGSTGSQQSEGSKSSGSTRSSRRAPGREKERRAAGAGGSGSESDHTAPSGVGSSWRERPAGQLSRGSSPRSQASATAPGLPPPHPTTKAYTVVGGPPGGPPVRELAAVPPELTGSRQSFQKAMGNPCEFFVDIM",
# STX16_HUMAN
"MATRRLTDAFLLLRNNSIQNRQLLAEQVSSHITSSPLHSRSIAAELDELADDRMALVSGISLDPEAAIGVTKRPPPKWVDGVDEIQYDVGRIKQKMKELASLHDKHLNRPTLDDSSEEEHAIEITTQEITQLFHRCQRAVQALPSRARACSEQEGRLLGNVVASLAQALQELSTSFRHAQSGYLKRMKNREERSQHFFDTSVPLMDDGDDNTLYHRGFTEDQLVLVEQNTLMVEEREREIRQIVQSISDLNEIFRDLGAMIVEQGTVLDRIDYNVEQSCIKTEDGLKQLHKAEQYQKKNRKMLVILILFVIIIVLIVVLVGVKSR",
# ADA10_HUMAN
"MVLLRVLILLLSWAAGMGGQYGNPLNKYIRHYEGLSYNVDSLHQKHQRAKRAVSHEDQFLRLDFHAHGRHFNLRMKRDTSLFSDEFKVETSNKVLDYDTSHIYTGHIYGEEGSFSHGSVIDGRFEGFIQTRGGTFYVEPAERYIKDRTLPFHSVIYHEDDINYPHKYGPQGGCADHSVFERMRKYQMTGVEEVTQIPQEEHAANGPELLRKKRTTSAEKNTCQLYIQTDHLFFKYYGTREAVIAQISSHVKAIDTIYQTTDFSGIRNISFMVKRIRINTTADEKDPTNPFRFPNIGVEKFLELNSEQNHDDYCLAYVFTDRDFDDGVLGLAWVGAPSGSSGGICEKSKLYSDGKKKSLNTGIITVQNYGSHVPPKVSHITFAHEVGHNFGSPHDSGTECTPGESKNLGQKENGNYIMYARATSGDKLNNNKFSLCSIRNISQVLEKKRNNCFVESGQPICGNGMVEQGEECDCGYSDQCKDECCFDANQPEGRKCKLKPGKQCSPSQGPCCTAQCAFKSKSEKCRDDSDCAREGICNGFTALCPASDPKPNFTDCNRHTQVCINGQCAGSICEKYGLEECTCASSDGKDDKELCHVCCMKKMDPSTCASTGSVQWSRHFSGRTITLQPGSPCNDFRGYCDVFMRCRLVDADGPLARLKKAIFSPELYENIAEWIVAHWWAVLLMGIALIMLMAGFIKICSVHTPSSNPKLPPPKPLPGTLKRRRPPQPIQQPQRQRPRESYQMGHMRR",
# GBRD_HUMAN
"MDAPARLLAPLLLLCAQQLRGTRAMNDIGDYVGSNLEISWLPNLDGLIAGYARNFRPGIGGPPVNVALALEVASIDHISEANMEYTMTVFLHQSWRDSRLSYNHTNETLGLDSRFVDKLWLPDTFIVNAKSAWFHDVTVENKLIRLQPDGVILYSIRITSTVACDMDLAKYPMDEQECMLDLESYGYSSEDIVYYWSESQEHIHGLDKLQLAQFTITSYRFTTELMNFKSAGQFPRLSLHFHLRRNRGVYIIQSYMPSVLLVAMSWVSFWISQAAVPARVSLGITTVLTMTTLMVSARSSLPRASAIKALDVYFWICYVFVFAALVEYAFAHFNADYRKKQKAKVKVSRPRAEMDVRNAIVLFSLSAAGVTQELAISRRQRRVPGNLMGSYRSVGVETGETKKEGAARSGGQGGIRARLRPIDADTIDIYARAVFPAAFAAVNVIYWAAYAM",
# GNB5_HUMAN
"MCDQTFLVNVFGSCDKCFKQRALRPVFKKSQQLSYCSTCAEIMATEGLHENETLASLKSEAESLKGKLEEERAKLHDVELHQVAERVEALGQFVMKTRRTLKGHGNKVLCMDWCKDKRRIVSSSQDGKVIVWDSFTTNKEHAVTMPCTWVMACAYAPSGCAIACGGLDNKCSVYPLTFDKNENMAAKKKSVAMHTNYLSACSFTNSDMQILTASGDGTCALWDVESGQLLQSFHGHGADVLCLDLAPSETGNTFVSGGCDKKAMVWDMRSGQCVQAFETHESDINSVRYYPSGDAFASGSDDATCRLYDLRADREVAIYSKESIIFGASSVDFSLSGRLLFAGYNDYTINVWDVLKGSRVSILFGHENRVSTLRVSPDGTAFCSGSWDHTLRVWA",
# NRP1_HUMAN
"MERGLPLLCAVLALVLAPAGAFRNDKCGDTIKIESPGYLTSPGYPHSYHPSEKCEWLIQAPDPYQRIMINFNPHFDLEDRDCKYDYVEVFDGENENGHFRGKFCGKIAPPPVVSSGPFLFIKFVSDYETHGAGFSIRYEIFKRGPECSQNYTTPSGVIKSPGFPEKYPNSLECTYIVFVPKMSEIILEFESFDLEPDSNPPGGMFCRYDRLEIWDGFPDVGPHIGRYCGQKTPGRIRSSSGILSMVFYTDSAIAKEGFSANYSVLQSSVSEDFKCMEALGMESGEIHSDQITASSQYSTNWSAERSRLNYPENGWTPGEDSYREWIQVDLGLLRFVTAVGTQGAISKETKKKYYVKTYKIDVSSNGEDWITIKEGNKPVLFQGNTNPTDVVVAVFPKPLITRFVRIKPATWETGISMRFEVYGCKITDYPCSGMLGMVSGLISDSQITSSNQGDRNWMPENIRLVTSRSGWALPPAPHSYINEWLQIDLGEEKIVRGIIIQGGKHRENKVFMRKFKIGYSNNGSDWKMIMDDSKRKAKSFEGNNNYDTPELRTFPALSTRFIRIYPERATHGGLGLRMELLGCEVEAPTAGPTTPNGNLVDECDDDQANCHSGTGDDFQLTGGTTVLATEKPTVIDSTIQSEFPTYGFNCEFGWGSHKTFCHWEHDNHVQLKWSVLTSKTGPIQDHTGDGNFIYSQADENQKGKVARLVSPVVYSQNSAHCMTFWYHMSGSHVGTLRVKLRYQKPEEYDQLVWMAIGHQGDHWKEGRVLLHKSLKLYQVIFEGEIGKGNLGGIAVDDISINNHISQEDCAKPADLDKKNPEIKIDETGSTPGYEGEGEGDKNISRKPGNVLKTLDPILITIIAMSALGVLLGAVCGVVLYCACWHNGMSERNLSALENYNFELVDGVKLKKDKLNTQSTYSEA",
# UN13B_HUMAN
"MSLLCVRVKRAKFQGSPDKFNTYVTLKVQNVKSTTVAVRGDQPSWEQDFMFEISRLDLGLSVEVWNKGLIWDTMVGTVWIALKTIRQSDEEGPGEWSTLEAETLMKDDEICGTRNPTPHKILLDTRFELPFDIPEEEARYWTYKWEQINALGADNEYSSQEESQRKPLPTAAAQCSFEDPDSAVDDRDSDYRSETSNSFPPPYHTASQPNASVHQFPVPVRSPQQLLLQGSSRDSCNDSMQSYDLDYPERRAISPTSSSRYGSSCNVSQGSSQLSELDQYHEQDDDHRETDSIHSCHSSHSLSRDGQAGFGEQEKPLEVTGQAEKEAACEPKEMKEDATTHPPPDLVLQKDHFLGPQESFPEENASSPFTQARAHWIRAVTKVRLQLQEIPDDGDPSLPQWLPEGPAGGLYGIDSMPDLRRKKPLPLVSDLSLVQSRKAGITSAMATRTSLKDEELKSHVYKKTLQALIYPISCTTPHNFEVWTATTPTYCYECEGLLWGIARQGMRCSECGVKCHEKCQDLLNADCLQRAAEKSCKHGAEDRTQNIIMAMKDRMKIRERNKPEIFEVIRDVFTVNKAAHVQQMKTVKQSVLDGTSKWSAKITITVVCAQGLQAKDKTGSSDPYVTVQVSKTKKRTKTIFGNLNPVWEEKFHFECHNSSDRIKVRVWDEDDDIKSRVKQRLKRESDDFLGQTIIEVRTLSGEMDVWYNLEKRTDKSAVSGAIRLQISVEIKGEEKVAPYHVQYTCLHENLFHYLTDIQGSGGVRIPEARGDDAWKVYFDETAQEIVDEFAMRYGIESIYQAMTHFACLSSKYMCPGVPAVMSTLLANINAYYAHTTASTNVSASDRFAASNFGKERFVKLLDQLHNSLRIDLSTYRNNFPAGSPERLQDLKSTVDLLTSITFFRMKVQELQSPPRASQVVKDCVKACLNSTYEYIFNNCHDLYSRQYQLKQELPPEEQGPSIRNLDFWPKLITLIVSIIEEDKNSYTPVLNQFPQELNVGKVSAEVMWHLFAQDMKYALEEHEKDHLCKSADYMNLHFKVKWLHNEYVRDLPVLQGQVPEYPAWFEQFVLQWLDENEDVSLEFLRGALERDKKDGFQQTSEHALFSCSVVDVFTQLNQSFEIIRKLECPDPSILAHYMRRFAKTIGKVLMQYADILSKDFPAYCTKEKLPCILMNNVQQLRVQLEKMFEAMGGKELDLEAADSLKELQVKLNTVLDELSMVFGNSFQVRIDECVRQMADILGQVRGTGNASPDARASAAQDADSVLRPLMDFLDGNLTLFATVCEKTVLKRVLKELWRVVMNTMERMIVLPPLTDQTGTQLIFTAAKELSHLSKLKDHMVREETRNLTPKQCAVLDLALDTIKQYFHAGGNGLKKTFLEKSPDLQSLRYALSLYTQTTDTLIKTFVRSQTTQGSGVDDPVGEVSIQVDLFTHPGTGEHKVTVKVVAANDLKWQTAGMFRPFVEVTMVGPHQSDKKRKFTTKSKSNNWAPKYNETFHFLLGNEEGPESYELQICVKDYCFAREDRVLGLAVMPLRDVTAKGSCACWCPLGRKIHMDETGLTILRILSQRSNDEVAREFVKLKSESRSTEEGS",
# CPLX1_HUMAN
"MEFVMKQALGGATKDMGKMLGGDEEKDPDAAKKEEERQEALRQAEEERKAKYAKMEAEREAVRQGIRDKYGIKKKEEREAEAQAAMEANSEGSLTRPKKAIPPGCGDEVEEEDESILDTVIKYLPGPLQDMLKK",
# RGRF2_HUMAN
"MQKSVRYNEGHALYLAFLARKEGTKRGFLSKKTAEASRWHEKWFALYQNVLFYFEGEQSCRPAGMYLLEGCSCERTPAPPRAGAGQGGVRDALDKQYYFTVLFGHEGQKPLELRCEEEQDGKEWMEAIHQASYADILIEREVLMQKYIHLVQIVETEKIAANQLRHQLEDQDTEIERLKSEIIALNKTKERMRPYQSNQEDEDPDIKKIKKVQSFMRGWLCRRKWKTIVQDYICSPHAESMRKRNQIVFTMVEAESEYVHQLYILVNGFLRPLRMAASSKKPPISHDDVSSIFLNSETIMFLHEIFHQGLKARIANWPTLILADLFDILLPMLNIYQEFVRNHQYSLQVLANCKQNRDFDKLLKQYEANPACEGRMLETFLTYPMFQIPRYIITLHELLAHTPHEHVERKSLEFAKSKLEELSRVMHDEVSDTENIRKNLAIERMIVEGCDILLDTSQTFIRQGSLIQVPSVERGKLSKVRLGSLSLKKEGERQCFLFTKHFLICTRSSGGKLHLLKTGGVLSLIDCTLIEEPDASDDDSKGSGQVFGHLDFKIVVEPPDAAAFTVVLLAPSRQEKAAWMSDISQCVDNIRCNGLMTIVFEENSKVTVPHMIKSDARLHKDDTDICFSKTLNSCKVPQIRYASVERLLERLTDLRFLSIDFLNTFLHTYRIFTTAAVVLGKLSDIYKRPFTSIPVRSLELFFATSQNNRGEHLVDGKSPRLCRKFSSPPPLAVSRTSSPVRARKLSLTSPLNSKIGALDLTTSSSPTTTTQSPAASPPPHTGQIPLDLSRGLSSPEQSPGTVEENVDNPRVDLCNKLKRSIQKAVLESAPADRAGVESSPAADTTELSPCRSPSTPRHLRYRQPGGQTADNAHCSVSPASAFAIATAAAGHGSPPGFNNTERTCDKEFIIRRTATNRVLNVLRHWVSKHAQDFELNNELKMNVLNLLEEVLRDPDLLPQERKAAANILRALSQDDQDDIHLKLEDIIQMTDCMKAECFESLSAMELAEQITLLDHVIFRSIPYEEFLGQGWMKLDKNERTPYIMKTSQHFNDMSNLVASQIMNYADVSSRANAIEKWVAVADICRCLHNYNGVLEITSALNRSAIYRLKKTWAKVSKQTKALMDKLQKTVSSEGRFKNLRETLKNCNPPAVPYLGMYLTDLAFIEEGTPNFTEEGLVNFSKMRMISHIIREIRQFQQTSYRIDHQPKVAQYLLDKDLIIDEDTLYELSLKIEPRLPA",
# GIPC1_HUMAN
"MPLGLGRRKKAPPLVENEEAEPGRGGLGVGEPGPLGGGGSGGPQMGLPPPPPALRPRLVFHTQLAHGSPTGRIEGFTNVKELYGKIAEAFRLPTAEVMFCTLNTHKVDMDKLLGGQIGLEDFIFAHVKGQRKEVEVFKSEDALGLTITDNGAGYAFIKRIKEGSVIDHIHLISVGDMIEAINGQSLLGCRHYEVARLLKELPRGRTFTLKLTEPRKAFDMISQRSAGGRPGSGPQLGTGRGTLRLRSRGPATVEDLPSAFEEKAIEKVDDLLESYMGIRDTELAATMVELGKDKRNPDELAEALDERLGDFAFPDEFVFDVWGAIGDAKVGRY",
# LIN7A_HUMAN
"MLKPSVTSAPTADMATLTVVQPLTLDRDVARAIELLEKLQESGEVPVHKLQSLKKVLQSEFCTAIREVYQYMHETITVNGCPEFRARATAKATVAAFAASEGHSHPRVVELPKTDEGLGFNVMGGKEQNSPIYISRIIPGGVAERHGGLKRGDQLLSVNGVSVEGEHHEKAVELLKAAKDSVKLVVRYTPKVLEEMEARFEKLRTARRRQQQQLLIQQQQQQQQQQTQQNHMS",
# PCD17_HUMAN
"MYLSICCCFLLWAPALTLKNLNYSVPEEQGAGTVIGNIGRDARLQPGLPPAERGGGGRSKSGSYRVLENSAPHLLDVDADSGLLYTKQRIDRESLCRHNAKCQLSLEVFANDKEICMIKVEIQDINDNAPSFSSDQIEMDISENAAPGTRFPLTSAHDPDAGENGLRTYLLTRDDHGLFGLDVKSRGDGTKFPELVIQKALDREQQNHHTLVLTALDGGEPPRSATVQINVKVIDSNDNSPVFEAPSYLVELPENAPLGTVVIDLNATDADEGPNGEVLYSFSSYVPDRVRELFSIDPKTGLIRVKGNLDYEENGMLEIDVQARDLGPNPIPAHCKVTVKLIDRNDNAPSIGFVSVRQGALSEAAPPGTVIALVRVTDRDSGKNGQLQCRVLGGGGTGGGGGLGGPGGSVPFKLEENYDNFYTVVTDRPLDRETQDEYNVTIVARDGGSPPLNSTKSFAIKILDENDNPPRFTKGLYVLQVHENNIPGEYLGSVLAQDPDLGQNGTVSYSILPSHIGDVSIYTYVSVNPTNGAIYALRSFNFEQTKAFEFKVLAKDSGAPAHLESNATVRVTVLDVNDNAPVIVLPTLQNDTAELQVPRNAGLGYLVSTVRALDSDFGESGRLTYEIVDGNDDHLFEIDPSSGEIRTLHPFWEDVTPVVELVVKVTDHGKPTLSAVAKLIIRSVSGSLPEGVPRVNGEQHHWDMSLPLIVTLSTISIILLAAMITIAVKCKRENKEIRTYNCRIAEYSHPQLGGGKGKKKKINKNDIMLVQSEVEERNAMNVMNVVSSPSLATSPMYFDYQTRLPLSSPRSEVMYLKPASNNLTVPQGHAGCHTSFTGQGTNASETPATRMSIIQTDNFPAEPNYMGSRQQFVQSSSTFKDPERASLRDSGHGDSDQADSDQDTNKGSCCDMSVREALKMKTTSTKSQPLEQEPEECVNCTDECRVLGHSDRCWMPQFPAANQAENADYRTNLFVPTVEANVETETYETVNPTGKKTFCTFGKDKREHTILIANVKPYLKAKRALSPLLQEVPSASSSPTKACIEPCTSTKGSLDGCEAKPGALAEASSQYLPTDSQYLSPSKQPRDPPFMASDQMARVFADVHSRASRDSSEMGAVLEQLDHPNRDLGRESVDAEEVVREIDKLLQDCRGNDPVAVRK",
# RGS12_HUMAN
"MFRAGEASKRPLPGPSPPRVRSVEVARGRAGYGFTLSGQAPCVLSCVMRGSPADFVGLRAGDQILAVNEINVKKASHEDVVKLIGKCSGVLHMVIAEGVGRFESCSSDEEGGLYEGKGWLKPKLDSKALGINRAERVVEEMQSGGIFNMIFENPSLCASNSEPLKLKQRSLSESAATRFDVGHESINNPNPNMLSKEEISKVIHDDSVFSIGLESHDDFALDASILNVAMIVGYLGSIELPSTSSNLESDSLQAIRGCMRRLRAEQKIHSLVTMKIMHDCVQLSTDKAGVVAEYPAEKLAFSAVCPDDRRFFGLVTMQTNDDGSLAQEEEGALRTSCHVFMVDPDLFNHKIHQGIARRFGFECTADPDTNGCLEFPASSLPVLQFISVLYRDMGELIEGMRARAFLDGDADAHQNNSTSSNSDSGIGNFHQEEKSNRVLVVDLGGSSSRHGPGGSAWDGVGGRGAQPWGAPWTGPFCPDPEGSPPFEAAHQTDRFWDLNKHLGPASPVEVPPASLRSSVPPSKRGTVGAGCGFNQRWLPVHVLREWQCGHTSDQDSYTDSTDGWSSINCGTLPPPMSKIPADRYRVEGSFAQPPLNAPKREWSRKAFGMQSIFGPHRNVRKTKEDKKGSKFGRGTGLTQPSQRTSARRSFGRSKRFSITRSLDDLESATVSDGELTGADLKDCVSNNSLSSNASLPSVQSCRRLRERRVASWAVSFERLLQDPVGVRYFSDFLRKEFSEENILFWQACEYFNHVPAHDKKELSYRAREIFSKFLCSKATTPVNIDSQAQLADDVLRAPHPDMFKEQQLQIFNLMKFDSYTRFLKSPLYQECILAEVEGRALPDSQQVPSSPASKHSLGSDHSSVSTPKKLSGKSKSGRSLNEELGDEDSEKKRKGAFFSWSRTRSTGRSQKKREHGDHADDALHANGGLCRRESQGSVSSAGSLDLSEACRTLAPEKDKATKHCCIHLPDGTSCVVAVKAGFSIKDILSGLCERHGINGAAADLFLVGGDKPLVLHQDSSILESRDLRLEKRTLFRLDLVPINRSVGLKAKPTKPVTEVLRPVVARYGLDLSGLLVRLSGEKEPLDLGAPISSLDGQRVVLEEKDPSRGKASADKQKGVPVKQNTAVNSSSRNHSATGEERTLGKSNSIKIKGENGKNARDPRLSKREESIAKIGKKKYQKINLDEAEEFFELISKAQSNRADDQRGLLRKEDLVLPEFLRLPPGSTELTLPTPAAVAKGFSKRSATGNGRESASQPGEQWEPVQESSDSPSTSPGSASSPPGPPGTTPPGQKSPSGPFCTPQSPVSLAQEGTAQIWKRQSQEVEAGGIQTVEDEHVAELTLMGEGDISSPNSTLLPPPSTPQEVPGPSRPGSGTHGSRDLPVNRIIDVDLVTGSAPGRDGGIAGAQAGPGRSQASGGPPTSDLPGLGPVPGEPAKPKTSAHHATFV",
# CSKP_HUMAN
"MADDDVLFEDVYELCEVIGKGPFSVVRRCINRETGQQFAVKIVDVAKFTSSPGLSTEDLKREASICHMLKHPHIVELLETYSSDGMLYMVFEFMDGADLCFEIVKRADAGFVYSEAVASHYMRQILEALRYCHDNNIIHRDVKPHCVLLASKENSAPVKLGGFGVAIQLGESGLVAGGRVGTPHFMAPEVVKREPYGKPVDVWGCGVILFILLSGCLPFYGTKERLFEGIIKGKYKMNPRQWSHISESAKDLVRRMLMLDPAERITVYEALNHPWLKERDRYAYKIHLPETVEQLRKFNARRKLKGAVLAAVSSHKFNSFYGDPPEELPDFSEDPTSSGLLAAERAVSQVLDSLEEIHALTDCSEKDLDFLHSVFQDQHLHTLLDLYDKINTKSSPQIRNPPSDAVQRAKEVLEEISCYPENNDAKELKRILTQPHFMALLQTHDVVAHEVYSDEALRVTPPPTSPYLNGDSPESANGDMDMENVTRVRLVQFQKNTDEPMGITLKMNELNHCIVARIMHGGMIHRQGTLHVGDEIREINGISVANQTVEQLQKMLREMRGSITFKIVPSYRTQSSSCERDSPSTSRQSPANGHSSTNNSVSDLPSTTQPKGRQIYVRAQFEYDPAKDDLIPCKEAGIRFRVGDIIQIISKDDHNWWQGKLENSKNGTAGLIPSPELQEWRVACIAMEKTKQEQQASCTWFGKKKKQYKDKYLAKHNAVFDQLDLVTYEEVVKLPAFKRKTLVLLGAHGVGRRHIKNTLITKHPDRFAYPIPHTTRPPKKDEENGKNYYFVSHDQMMQDISNNEYLEYGSHEDAMYGTKLETIRKIHEQGLIAILDVEPQALKVLRTAEFAPFVVFIAAPTITPGLNEDESLQRLQKESDILQRTYAHYFDLTIINNEIDETIRHLEEAVELVCTAPQWVPVSWVY",
# AURKA_HUMAN
"MDRSKENCISGPVKATAPVGGPKRVLVTQQFPCQNPLPVNSGQAQRVLCPSNSSQRIPLQAQKLVSSHKPVQNQKQKQLQATSVPHPVSRPLNNTQKSKQPLPSAPENNPEEELASKQKNEESKKRQWALEDFEIGRPLGKGKFGNVYLAREKQSKFILALKVLFKAQLEKAGVEHQLRREVEIQSHLRHPNILRLYGYFHDATRVYLILEYAPLGTVYRELQKLSKFDEQRTATYITELANALSYCHSKRVIHRDIKPENLLLGSAGELKIADFGWSVHAPSSRRTTLCGTLDYLPPEMIEGRMHDEKVDLWSLGVLCYEFLVGKPPFEANTYQETYKRISRVEFTFPDFVTEGARDLISRLLKHNPSQRPMLREVLEHPWITANSSKPSNCQNKESASKQS",
# HNRDL_HUMAN
"MEVPPRLSHVPPPLFPSAPATLASRSLSHWRPRPPRQLAPLLPSLAPSSARQGARRAQRHVTAQQPSRLAGGAAIKGGRRRRPDLFRRHFKSSSIQRSAAAAAATRTARQHPPADSSVTMEDMNEYSNIEEFAEGSKINASKNQQDDGKMFIGGLSWDTSKKDLTEYLSRFGEVVDCTIKTDPVTGRSRGFGFVLFKDAASVDKVLELKEHKLDGKLIDPKRAKALKGKEPPKKVFVGGLSPDTSEEQIKEYFGAFGEIENIELPMDTKTNERRGFCFITYTDEEPVKKLLESRYHQIGSGKCEIKVAQPKEVYRQQQQQQKGGRGAAAGGRGGTRGRGRGQGQNWNQGFNNYYDQGYGNYNSAYGGDQNYSGYGGYDYTGYNYGNYGYGQGYADYSGQQSTYGKASRGGGNHQNNYQPY",
# SYN3_HUMAN
"MNFLRRRLSDSSFMANLPNGYMTDLQRPDSSTSSPASPAMERRHPQPLAASFSSPGSSLFSSLSSAMKQAPQATSGLMEPPGPSTPIVQRPRILLVIDDAHTDWSKYFHGKKVNGEIEIRVEQAEFSELNLAAYVTGGCMVDMQVVRNGTKVVSRSFKPDFILVRQHAYSMALGEDYRSLVIGLQYGGLPAVNSLYSVYNFCSKPWVFSQLIKIFHSLGPEKFPLVEQTFFPNHKPMVTAPHFPVVVKLGHAHAGMGKIKVENQLDFQDITSVVAMAKTYATTEAFIDSKYDIRIQKIGSNYKAYMRTSISGNWKANTGSAMLEQVAMTERYRLWVDSCSEMFGGLDICAVKAVHSKDGRDYIIEVMDSSMPLIGEHVEEDRQLMADLVVSKMSQLPMPGGTAPSPLRPWAPQIKSAKSPGQAQLGPQLGQPQPRPPPQGGPRQAQSPQPQRSGSPSQQRLSPQGQQPLSPQSGSPQQQRSPGSPQLSRASSGSSPNQASKPGATLASQPRPPVQGRSTSQQGEESKKPAPPHPHLNKSQSLTNSLSTSDTSQRGTPSEDEAKAETIRNLRKSFASLFSD",
# SPTN2_HUMAN
"MSSTLSPTDFDSLEIQGQYSDINNRWDLPDSDWDNDSSSARLFERSRIKALADEREAVQKKTFTKWVNSHLARVTCRVGDLYSDLRDGRNLLRLLEVLSGEILPKPTKGRMRIHCLENVDKALQFLKEQKVHLENMGSHDIVDGNHRLTLGLVWTIILRFQIQDISVETEDNKEKKSAKDALLLWCQMKTAGYPNVNVHNFTTSWRDGLAFNAIVHKHRPDLLDFESLKKCNAHYNLQNAFNLAEKELGLTKLLDPEDVNVDQPDEKSIITYVATYYHYFSKMKALAVEGKRIGKVLDHAMEAERLVEKYESLASELLQWIEQTIVTLNDRQLANSLSGVQNQLQSFNSYRTVEKPPKFTEKGNLEVLLFTIQSKLRANNQKVYTPREGRLISDINKAWERLEKAEHERELALRTELIRQEKLEQLAARFDRKAAMRETWLSENQRLVSQDNFGLELAAVEAAVRKHEAIETDIVAYSGRVQAVDAVAAELAAERYHDIKRIAARQHNVARLWDFLRQMVAARRERLLLNLELQKVFQDLLYLMDWMEEMKGRLQSQDLGRHLAGVEDLLQLHELVEADIAVQAERVRAVSASALRFCNPGKEYRPCDPQLVSERVAKLEQSYEALCELAAARRARLEESRRLWRFLWEVGEAEAWVREQQHLLASADTGRDLTGALRLLNKHTALRGEMSGRLGPLKLTLEQGQQLVAEGHPGASQASARAAELQAQWERLEALAEERAQRLAQAASLYQFQADANDMEAWLVDALRLVSSPELGHDEFSTQALARQHRALEEEIRSHRPTLDALREQAAALPPTLSRTPEVQSRVPTLERHYEELQARAGERARALEAALALYTMLSEAGACGLWVEEKEQWLNGLALPERLEDLEVVQQRFETLEPEMNTLAAQITAVNDIAEQLLKANPPGKDRIVNTQEQLNHRWQQFRRLADGKKAALTSALSIQNYHLECTETQAWMREKTKVIESTQGLGNDLAGVLALQRKLAGTERDLEAIAARVGELTREANALAAGHPAQAVAINARLREVQTGWEDLRATMRRREESLGEARRLQDFLRSLDDFQAWLGRTQTAVASEEGPATLPEAEALLAQHAALRGEVERAQSEYSRLRALGEEVTRDQADPQCLFLRQRLEALGTGWEELGRMWESRQGRLAQAHGFQGFLRDARQAEGVLSSQEYVLSHTEMPGTLQAADAAIKKLEDFMSTMDANGERIHGLLEAGRQLVSEGNIHADKIREKADSIERRHKKNQDAAQQFLGRLRDNREQQHFLQDCHELKLWIDEKMLTAQDVSYDEARNLHTKWQKHQAFMAELAANKDWLDKVDKEGRELTLEKPELKALVSEKLRDLHRRWDELETTTQAKARSLFDANRAELFAQSCCALESWLESLQAQLHSDDYGKDLTSVNILLKKQQMLEWEMAVREKEVEAIQAQAKALAQEDQGAGEVERTSRAVEEKFRALCQPMRERCRRLQASREQHQFHRDVEDEILWVTERLPMASSMEHGKDLPSVQLLMKKNQTLQKEIQGHEPRIADLRERQRALGAAAAGPELAELQEMWKRLGHELELRGKRLEDALRAQQFYRDAAEAEAWMGEQELHMMGQEKAKDELSAQAEVKKHQVLEQALADYAQTIHQLAASSQDMIDHEHPESTRISIRQAQVDKLYAGLKELAGERRERLQEHLRLCQLRRELDDLEQWIQEREVVAASHELGQDYEHVTMLRDKFREFSRDTSTIGQERVDSANALANGLIAGGHAARATVAEWKDSLNEAWADLLELLDTRGQVLAAAYELQRFLHGARQALARVQHKQQQLPDGTGRDLNAAEALQRRHCAYEHDIQALSPQVQQVQDDGHRLQKAYAGDKAEEIGRHMQAVAEAWAQLQGSSAARRQLLLDTTDKFRFFKAVRELMLWMDEVNLQMDAQERPRDVSSADLVIKNQQGIKAEIEARADRFSSCIDMGKELLARSHYAAEEISEKLSQLQARRQETAEKWQEKMDWLQLVLEVLVFGRDAGMAEAWLCSQEPLVRSAELGCTVDEVESLIKRHEAFQKSAVAWEERFCALEKLTALEEREKERKRKREEEERRKQPPAPEPTASVPPGDLVGGQTASDTTWDGTQPRPPPSTQAPSVNGVCTDGEPSQPLLGQQRLEHSSFPEGPGPGSGDEANGPRGERQTRTRGPAPSAMPQSRSTESAHAATLPPRGPEPSAQEQMEGMLCRKQEMEAFGKKAANRSWQNVYCVLRRGSLGFYKDAKAASAGVPYHGEVPVSLARAQGSVAFDYRKRKHVFKLGLQDGKEYLFQAKDEAEMSSWLRVVNAAIATASSASGEPEEPVVPSTTRGMTRAMTMPPVSPVGAEGPVVLRSKDGREREREKRFSFFKKNK",
# RIMB2_HUMAN
"MREAAERRQQLQLEHDQALAVLSAKQQEIDLLQKSKVRELEEKCRTQSEQFNLLSRDLEKFRQHAGKIDLLGGSAVAPLDISTAPSKPFPQFMNGLATSLGKGQESAIGGSSAIGEYIRPLPQPGDRPEPLSAKPTFLSRSGSARCRSESDMENERNSNTSKQRYSGKVHLCVARYSYNPFDGPNENPEAELPLTAGKYLYVYGDMDEDGFYEGELLDGQRGLVPSNFVDFVQDNESRLASTLGNEQDQNFINHSGIGLEGEHILDLHSPTHIDAGITDNSAGTLDVNIDDIGEDIVPYPRKITLIKQLAKSVIVGWEPPAVPPGWGTVSSYNVLVDKETRMNLTLGSRTKALIEKLNMAACTYRISVQCVTSRGSSDELQCTLLVGKDVVVAPSHLRVDNITQISAQLSWLPTNSNYSHVIFLNEEEFDIVKAARYKYQFFNLRPNMAYKVKVLAKPHQMPWQLPLEQREKKEAFVEFSTLPAGPPAPPQDVTVQAGVTPATIRVSWRPPVLTPTGLSNGANVTGYGVYAKGQRVAEVIFPTADSTAVELVRLRSLEAKGVTVRTLSAQGESVDSAVAAVPPELLVPPTPHPRPAPQSKPLASSGVPETKDEHLGPHARMDEAWEQSRAPGPVHGHMLEPPVGPGRRSPSPSRILPQPQGTPVSTTVAKAMAREAAQRVAESSRLEKRSVFLERSSAGQYAASDEEDAYDSPDFKRRGASVDDFLKGSELGKQPHCCHGDEYHTESSRGSDLSDIMEEDEEELYSEMQLEDGGRRRPSGTSHNALKILGNPASAGRVDHMGRRFPRGSAGPQRSRPVTVPSIDDYGRDRLSPDFYEESETDPGAEELPARIFVALFDYDPLTMSPNPDAAEEELPFKEGQIIKVYGDKDADGFYRGETCARLGLIPCNMVSEIQADDEEMMDQLLRQGFLPLNTPVEKIERSRRSGRRHSVSTRRMVALYDYDPRESSPNVDVEAELTFCTGDIITVFGEIDEDGFYYGELNGQKGLVPSNFLEEVPDDVEVYLSDAPSHYSQDTPMRSKAKRKKSVHFTP",
# SYNJ2_HUMAN
"MALSKGLRLLGRLGAEGDCSVLLEARGRDDCLLFEAGTVATLAPEEKEVIKGQYGKLTDAYGCLGELRLKSGGTSLSFLVLVTGCTSVGRIPDAEIYKITATDFYPLQEEAKEEERLIALKKILSSGVFYFSWPNDGSRFDLTVRTQKQGDDSSEWGNSFFWNQLLHVPLRQHQVSCCDWLLKIICGVVTIRTVYASHKQAKACLVSRVSCERTGTRFHTRGVNDDGHVSNFVETEQMIYMDDGVSSFVQIRGSVPLFWEQPGLQVGSHHLRLHRGLEANAPAFDRHMVLLKEQYGQQVVVNLLGSRGGEEVLNRAFKKLLWASCHAGDTPMINFDFHQFAKGGKLEKLETLLRPQLKLHWEDFDVFTKGENVSPRFQKGTLRMNCLDCLDRTNTVQSFIALEVLHLQLKTLGLSSKPIVDRFVESFKAMWSLNGHSLSKVFTGSRALEGKAKVGKLKDGARSMSRTIQSNFFDGVKQEAIKLLLVGDVYGEEVADKGGMLLDSTALLVTPRILKAMTERQSEFTNFKRIRIAMGTWNVNGGKQFRSNVLRTAELTDWLLDSPQLSGATDSQDDSSPADIFAVGFEEMVELSAGNIVNASTTNKKMWGEQLQKAISRSHRYILLTSAQLVGVCLYIFVRPYHVPFIRDVAIDTVKTGMGGKAGNKGAVGIRFQFHSTSFCFICSHLTAGQSQVKERNEDYKEITQKLCFPMGRNVFSHDYVFWCGDFNYRIDLTYEEVFYFVKRQDWKKLLEFDQLQLQKSSGKIFKDFHEGAINFGPTYKYDVGSAAYDTSDKCRTPAWTDRVLWWRKKHPFDKTAGELNLLDSDLDVDTKVRHTWSPGALQYYGRAELQASDHRPVLAIVEVEVQEVDVGARERVFQEVSSFQGPLDATVVVNLQSPTLEEKNEFPEDLRTELMQTLGSYGTIVLVRINQGQMLVTFADSHSALSVLDVDGMKVKGRAVKIRPKTKDWLKGLREEIIRKRDSMAPVSPTANSCLLEENFDFTSLDYESEGDILEDDEDYLVDEFNQPGVSDSELGGDDLSDVPGPTALAPPSKSPALTKKKQHPTYKDDADLVELKRELEAVGEFRHRSPSRSLSVPNRPRPPQPPQRPPPPTGLMVKKSASDASISSGTHGQYSILQTARLLPGAPQQPPKARTGISKPYNVKQIKTTNAQEAEAAIRCLLEARGGASEEALSAVAPRDLEASSEPEPTPGAAKPETPQAPPLLPRRPPPRVPAIKKPTLRRTGKPLSPEEQFEQQTVHFTIGPPETSVEAPPVVTAPRVPPVPKPRTFQPGKAAERPSHRKPASDEAPPGAGASVPPPLEAPPLVPKVPPRRKKSAPAAFHLQVLQSNSQLLQGLTYNSSDSPSGHPPAAGTVFPQGDFLSTSSATSPDSDGTKAMKPEAAPLLGDYQDPFWNLLHHPKLLNNTWLSKSSDPLDSGTRSPKRDPIDPVSAGASAAKAELPPDHEHKTLGHWVTISDQEKRTALQVFDPLAKT",
# KIF3B_HUMAN
"MSKLKSSESVRVVVRCRPMNGKEKAASYDKVVDVDVKLGQVSVKNPKGTAHEMPKTFTFDAVYDWNAKQFELYDETFRPLVDSVLQGFNGTIFAYGQTGTGKTYTMEGIRGDPEKRGVIPNSFDHIFTHISRSQNQQYLVRASYLEIYQEEIRDLLSKDQTKRLELKERPDTGVYVKDLSSFVTKSVKEIEHVMNVGNQNRSVGATNMNEHSSRSHAIFVITIECSEVGLDGENHIRVGKLNLVDLAGSERQAKTGAQGERLKEATKINLSLSALGNVISALVDGKSTHIPYRDSKLTRLLQDSLGGNAKTVMVANVGPASYNVEETLTTLRYANRAKNIKNKPRVNEDPKDALLREFQEEIARLKAQLEKRSIGRRKRREKRREGGGSGGGGEEEEEEGEEGEEEGDDKDDYWREQQEKLEIEKRAIVEDHSLVAEEKMRLLKEKEKKMEDLRREKDAAEMLGAKIKAMESKLLVGGKNIVDHTNEQQKILEQKRQEIAEQKRREREIQQQMESRDEETLELKETYSSLQQEVDIKTKKLKKLFSKLQAVKAEIHDLQEEHIKERQELEQTQNELTRELKLKHLIIENFIPLEEKSKIMNRAFFDEEEDHWKLHPITRLENQQMMKRPVSAVGYKRPLSQHARMSMMIRPEARYRAENIVLLELDMPSRTTRDYEGPAIAPKVQAALDAALQDEDEIQVDASSFESTANKKSKARPKSGRKSGSSSSSSGTPASQLYPQSRGLVPK",
# DCLK1_HUMAN
"MSFGRDMELEHFDERDKAQRYSRGSRVNGLPSPTHSAHCSFYRTRTLQTLSSEKKAKKVRFYRNGDRYFKGIVYAISPDRFRSFEALLADLTRTLSDNVNLPQGVRTIYTIDGLKKISSLDQLVEGESYVCGSIEPFKKLEYTKNVNPNWSVNVKTTSASRAVSSLATAKGSPSEVRENKDFIRPKLVTIIRSGVKPRKAVRILLNKKTAHSFEQVLTDITDAIKLDSGVVKRLYTLDGKQVMCLQDFFGDDDIFIACGPEKFRYQDDFLLDESECRVVKSTSYTKIASSSRRSTTKSPGPSRRSKSPASTSSVNGTPGSQLSTPRSGKSPSPSPTSPGSLRKQRSSQHGGSSTSLASTKVCSSMDENDGPGEEVSEEGFQIPATITERYKVGRTIGDGNFAVVKECVERSTAREYALKIIKKSKCRGKEHMIQNEVSILRRVKHPNIVLLIEEMDVPTELYLVMELVKGGDLFDAITSTNKYTERDASGMLYNLASAIKYLHSLNIVHRDIKPENLLVYEHQDGSKSLKLGDFGLATIVDGPLYTVCGTPTYVAPEIIAETGYGLKVDIWAAGVITYILLCGFPPFRGSGDDQEVLFDQILMGQVDFPSPYWDNVSDSAKELITMMLLVDVDQRFSAVQVLEHPWVNDDGLPENEHQLSVAGKIKKHFNTGPKPNSTAAGVSVIATTALDKERQVFRRRRNQDVRSRYKAQPAPPELNSESEDYSPSSSETVRSPNSPF",
# SNPH_HUMAN
"MAMSLPGSRRTSAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSYKGSDSSPTPRRSMKYTLCSDNHGIKPPTPEQYLTPLQQKEVCIRHLKARLKDTQDRLQDRDTEIDDLKTQLSRMQEDWIEEECHRVEAQLALKEARKEIKQLKQVIDTVKNNLIDKDKGLQKYFVDINIQNKKLETLLHSMEVAQNGMAKEDGTGESAGGSPARSLTRSSTYTKLSDPAVCGDRQPGDPSSGSAEDGADSGFAAADDTLSRTDALEASSLLSSGVDCGTEETSLHSSFGLGPRFPASNTYEKLLCGMEAGVQASCMQERAIQTDFVQYQPDLDTILEKVTQAQVCGTDPESGDRCPELDAHPSGPRDPNSAVVVTVGDELEAPEPITRGPTPQRPGANPNPGQSVSVVCPMEEEEEAAVAEKEPKSYWSRHYIVDLLAVVVPAVPTVAWLCRSQRRQGQPIYNISSLLRGCCTVALHSIRRISCRSLSQPSPSPAGGGSQL",
# ERC2_HUMAN
"MYGSARTITNLEGSPSRSPRLPRSPRLGHRRTSSGGGGGTGKTLSMENIQSLNAAYATSGPMYLSDHEGVASTTYPKGTMTLGRATNRAVYGGRVTAMGSSPNIASAGLSHTDVLSYTDQHGGLTGSSHHHHHQVPSMLRQVRDSTMLDLQAQLKELQRENDLLRKELDIKDSKLGSSMNSIKTFWSPELKKERVLRKEEAARMSVLKEQMRVSHEENQHLQLTIQALQDELRTQRDLNHLLQQESGNRGAEHFTIELTEENFRRLQAEHDRQAKELFLLRKTLEEMELRIETQKQTLNARDESIKKLLEMLQSKGLPSKSLEDDNERTRRMAEAESQVSHLEVILDQKEKENIHLREELHRRSQLQPEPAKTKALQTVIEMKDTKIASLERNIRDLEDEIQMLKANGVLNTEDREEEIKQIEVYKSHSKFMKTKIDQLKQELSKKESELLALQTKLETLSNQNSDCKQHIEVLKESLTAKEQRAAILQTEVDALRLRLEEKESFLNKKTKQLQDLTEEKGTLAGEIRDMKDMLEVKERKINVLQKKIENLQEQLRDKDKQLTNLKDRVKSLQTDSSNTDTALATLEEALSEKERIIERLKEQRERDDRERLEEIESFRKENKDLKEKVNALQAELTEKESSLIDLKEHASSLASAGLKRDSKLKSLEIAIEQKKEECSKLEAQLKKAHNIEDDSRMNPEFADQIKQLDKEASYYRDECGKAQAEVDRLLEILKEVENEKNDKDKKIAELESLTLRHMKDQNKKVANLKHNQQLEKKKNAQLLEEVRRREDSMADNSQHLQIEELMNALEKTRQELDATKARLASTQQSLAEKEAHLANLRIERRKQLEEILEMKQEALLAAISEKDANIALLELSASKKKKTQEEVMALKREKDRLVHQLKQQTQNRMKLMADNYDDDHHHYHHHHHHHHHRSPGRSQHSNHRPSPDQDDEEGIWA",
# SCAM1_HUMAN
"MSDFDSNPFADPDLNNPFKDPSVTQVTRNVPPGLDEYNPFSDSRTPPPGGVKMPNVPNTQPAIMKPTEEHPAYTQIAKEHALAQAELLKRQEELERKAAELDRREREMQNLSQHGRKNNWPPLPSNFPVGPCFYQDFSVDIPVEFQKTVKLMYYLWMFHAVTLFLNIFGCLAWFCVDSARAVDFGLSILWFLLFTPCSFVCWYRPLYGAFRSDSSFRFFVFFFVYICQFAVHVLQAAGFHNWGNCGWISSLTGLNQNIPVGIMMIIIAALFTASAVISLVMFKKVHGLYRTTGASFEKAQQEFATGVMSNKTVQTAAANAASTAASSAAQNAFKGNQI",
# NPFF_HUMAN
"MDSRQAAALLVLLLLIDGGCAEGPGGQQEDQLSAEEDSEPLPPQDAQTSGSLLHYLLQAMERPGRSQAFLFQPQRFGRNTQGSWRNEWLSPRAGEGLNSQFWSLAAPQRFGKK",
# ARPC2_HUMAN
"MILLEVNNRIIEETLALKFENAAAGNKPEAVEVTFADFDGVLYHISNPNGDKTKVMVSISLKFYKELQAHGADELLKRVYGSFLVNPESGYNVSLLYDLENLPASKDSIVHQAGMLKRNCFASVFEKYFQFQEEGKEGENRAVIHYRDDETMYVESKKDRVTVVFSTVFKDDDDVVIGKVFMQEFKEGRRASHTAPQVLFSHREPPLELKDTDAAVGDNIGYITFVLFPRHTNASARDNTINLIHTFRDYLHYHIKCSKAYIHTRMRAKTSDFLKVLNRARPDAEKKEMKTITGKTFSSR",
# ARPC3_HUMAN
"MPAYHSSLMDPDTKLIGNMALLPIRSQFKGPAPRETKDTDIVDEAIYYFKANVFFKNYEIKNEADRTLIYITLYISECLKKLQKCNSKSQGEKEMYTLGITNFPIPGEPGFPLNAIYAKPANKQEDEVMRAYLQQLRQETGLRLCEKVFDPQNDKPSKWWTCFVKRQFMNKSLSGPGQ",
# AXIN1_HUMAN
"MNIQEQGFPLDLGASFTEDAPRPPVPGEEGELVSTDPRPASYSFCSGKGVGIKGETSTATPRRSDLDLGYEPEGSASPTPPYLKWAESLHSLLDDQDGISLFRTFLKQEGCADLLDFWFACTGFRKLEPCDSNEEKRLKLARAIYRKYILDNNGIVSRQTKPATKSFIKGCIMKQLIDPAMFDQAQTEIQATMEENTYPSFLKSDIYLEYTRTGSESPKVCSDQSSGSGTGKGISGYLPTLNEDEEWKCDQDMDEDDGRDAAPPGRLPQKLLLETAAPRVSSSRRYSEGREFRYGSWREPVNPYYVNAGYALAPATSANDSEQQSLSSDADTLSLTDSSVDGIPPYRIRKQHRREMQESVQVNGRVPLPHIPRTYRVPKEVRVEPQKFAEELIHRLEAVQRTREAEEKLEERLKRVRMEEEGEDGDPSSGPPGPCHKLPPAPAWHHFPPRCVDMGCAGLRDAHEENPESILDEHVQRVLRTPGRQSPGPGHRSPDSGHVAKMPVALGGAASGHGKHVPKSGAKLDAAGLHHHRHVHHHVHHSTARPKEQVEAEATRRAQSSFAWGLEPHSHGARSRGYSESVGAAPNASDGLAHSGKVGVACKRNAKKAESGKSASTEVPGASEDAEKNQKIMQWIIEGEKEISRHRRTGHGSSGTRKPQPHENSRPLSLEHPWAGPQLRTSVQPSHLFIQDPTMPPHPAPNPLTQLEEARRRLEEEEKRASRAPSKQRYVQEVMRRGRACVRPACAPVLHVVPAVSDMELSETETRSQRKVGGGSAQPCDSIVVAYYFCGEPIPYRTLVRGRAVTLGQFKELLTKKGSYRYYFKKVSDEFDCGVVFEEVREDEAVLPVFEEKIIGKVEKVD",
# PGRC2_HUMAN
"MAAGDGDVKLGTLGSGSESSNDGGSESPGDAGAAAEGGGWAAAALALLTGGGEMLLNVALVALVLLGAYRLWVRWGRRGLGAGAGAGEESPATSLPRMKKRDFSLEQLRQYDGSRNPRILLAVNGKVFDVTKGSKFYGPAGPYGIFAGRDASRGLATFCLDKDALRDEYDDLSDLNAVQMESVREWEMQFKEKYDYVGRLLKPGEEPSEYTDEEDTKDHNKQD",
# LAMA5_HUMAN
"MAKRLCAGSALCVRGPRGPAPLLLVGLALLGAARAREEAGGGFSLHPPYFNLAEGARIAASATCGEEAPARGSPRPTEDLYCKLVGGPVAGGDPNQTIRGQYCDICTAANSNKAHPASNAIDGTERWWQSPPLSRGLEYNEVNVTLDLGQVFHVAYVLIKFANSPRPDLWVLERSMDFGRTYQPWQFFASSKRDCLERFGPQTLERITRDDAAICTTEYSRIVPLENGEIVVSLVNGRPGAMNFSYSPLLREFTKATNVRLRFLRTNTLLGHLMGKALRDPTVTRRYYYSIKDISIGGRCVCHGHADACDAKDPTDPFRLQCTCQHNTCGGTCDRCCPGFNQQPWKPATANSANECQSCNCYGHATDCYYDPEVDRRRASQSLDGTYQGGGVCIDCQHHTTGVNCERCLPGFYRSPNHPLDSPHVCRRCNCESDFTDGTCEDLTGRCYCRPNFSGERCDVCAEGFTGFPSCYPTPSSSNDTREQVLPAGQIVNCDCSAAGTQGNACRKDPRVGRCLCKPNFQGTHCELCAPGFYGPGCQPCQCSSPGVADDRCDPDTGQCRCRVGFEGATCDRCAPGYFHFPLCQLCGCSPAGTLPEGCDEAGRCLCQPEFAGPHCDRCRPGYHGFPNCQACTCDPRGALDQLCGAGGLCRCRPGYTGTACQECSPGFHGFPSCVPCHCSAEGSLHAACDPRSGQCSCRPRVTGLRCDTCVPGAYNFPYCEAGSCHPAGLAPVDPALPEAQVPCMCRAHVEGPSCDRCKPGFWGLSPSNPEGCTRCSCDLRGTLGGVAECQPGTGQCFCKPHVCGQACASCKDGFFGLDQADYFGCRSCRCDIGGALGQSCEPRTGVCRCRPNTQGPTCSEPARDHYLPDLHHLRLELEEAATPEGHAVRFGFNPLEFENFSWRGYAQMAPVQPRIVARLNLTSPDLFWLVFRYVNRGAMSVSGRVSVREEGRSATCANCTAQSQPVAFPPSTEPAFITVPQRGFGEPFVLNPGTWALRVEAEGVLLDYVVLLPSAYYEAALLQLRVTEACTYRPSAQQSGDNCLLYTHLPLDGFPSAAGLEALCRQDNSLPRPCPTEQLSPSHPPLITCTGSDVDVQLQVAVPQPGRYALVVEYANEDARQEVGVAVHTPQRAPQQGLLSLHPCLYSTLCRGTARDTQDHLAVFHLDSEASVRLTAEQARFFLHGVTLVPIEEFSPEFVEPRVSCISSHGAFGPNSAACLPSRFPKPPQPIILRDCQVIPLPPGLPLTHAQDLTPAMSPAGPRPRPPTAVDPDAEPTLLREPQATVVFTTHVPTLGRYAFLLHGYQPAHPTFPVEVLINAGRVWQGHANASFCPHGYGCRTLVVCEGQALLDVTHSELTVTVRVPKGRWLWLDYVLVVPENVYSFGYLREEPLDKSYDFISHCAAQGYHISPSSSSLFCRNAAASLSLFYNNGARPCGCHEVGATGPTCEPFGGQCPCHAHVIGRDCSRCATGYWGFPNCRPCDCGARLCDELTGQCICPPRTIPPDCLLCQPQTFGCHPLVGCEECNCSGPGIQELTDPTCDTDSGQCKCRPNVTGRRCDTCSPGFHGYPRCRPCDCHEAGTAPGVCDPLTGQCYCKENVQGPKCDQCSLGTFSLDAANPKGCTRCFCFGATERCRSSSYTRQEFVDMEGWVLLSTDRQVVPHERQPGTEMLRADLRHVPEAVPEAFPELYWQAPPSYLGDRVSSYGGTLRYELHSETQRGDVFVPMESRPDVVLQGNQMSITFLEPAYPTPGHVHRGQLQLVEGNFRHTETRNTVSREELMMVLASLEQLQIRALFSQISSAVFLRRVALEVASPAGQGALASNVELCLCPASYRGDSCQECAPGFYRDVKGLFLGRCVPCQCHGHSDRCLPGSGVCVDCQHNTEGAHCERCQAGFVSSRDDPSAPCVSCPCPLSVPSNNFAEGCVLRGGRTQCLCKPGYAGASCERCAPGFFGNPLVLGSSCQPCDCSGNGDPNLLFSDCDPLTGACRGCLRHTTGPRCEICAPGFYGNALLPGNCTRCDCTPCGTEACDPHSGHCLCKAGVTGRRCDRCQEGHFGFDGCGGCRPCACGPAAEGSECHPQSGQCHCRPGTMGPQCRECAPGYWGLPEQGCRRCQCPGGRCDPHTGRCNCPPGLSGERCDTCSQQHQVPVPGGPVGHSIHCEVCDHCVVLLLDDLERAGALLPAIHEQLRGINASSMAWARLHRLNASIADLQSQLRSPLGPRHETAQQLEVLEQQSTSLGQDARRLGGQAVGTRDQASQLLAGTEATLGHAKTLLAAIRAVDRTLSELMSQTGHLGLANASAPSGEQLLRTLAEVERLLWEMRARDLGAPQAAAEAELAAAQRLLARVQEQLSSLWEENQALATQTRDRLAQHEAGLMDLREALNRAVDATREAQELNSRNQERLEEALQRKQELSRDNATLQATLHAARDTLASVFRLLHSLDQAKEELERLAASLDGARTPLLQRMQTFSPAGSKLRLVEAAEAHAQQLGQLALNLSSIILDVNQDRLTQRAIEASNAYSRILQAVQAAEDAAGQALQQADHTWATVVRQGLVDRAQQLLANSTALEEAMLQEQQRLGLVWAALQGARTQLRDVRAKKDQLEAHIQAAQAMLAMDTDETSKKIAHAKAVAAEAQDTATRVQSQLQAMQENVERWQGQYEGLRGQDLGQAVLDAGHSVSTLEKTLPQLLAKLSILENRGVHNASLALSASIGRVRELIAQARGAASKVKVPMKFNGRSGVQLRTPRDLADLAAYTALKFYLQGPEPEPGQGTEDRFVMYMGSRQATGDYMGVSLRDKKVHWVYQLGEAGPAVLSIDEDIGEQFAAVSLDRTLQFGHMSVTVERQMIQETKGDTVAPGAEGLLNLRPDDFVFYVGGYPSTFTPPPLLRFPGYRGCIEMDTLNEEVVSLYNFERTFQLDTAVDRPCARSKSTGDPWLTDGSYLDGTGFARISFDSQISTTKRFEQELRLVSYSGVLFFLKQQSQFLCLAVQEGSLVLLYDFGAGLKKAVPLQPPPPLTSASKAIQVFLLGGSRKRVLVRVERATVYSVEQDNDLELADAYYLGGVPPDQLPPSLRRLFPTGGSVRGCVKGIKALGKYVDLKRLNTTGVSAGCTADLLVGRAMTFHGHGFLRLALSNVAPLTGNVYSGFGFHSAQDSALLYYRASPDGLCQVSLQQGRVSLQLLRTEVKTQAGFADGAPHYVAFYSNATGVWLYVDDQLQQMKPHRGPPPELQPQPEGPPRLLLGGLPESGTIYNFSGCISNVFVQRLLGPQRVFDLQQNLGSVNVSTGCAPALQAQTPGLGPRGLQATARKASRRSRQPARHPACMLPPHLRTTRDSYQFGGSLSSHLEFVGILARHRNWPSLSMHVLPRSSRGLLLFTARLRPGSPSLALFLSNGHFVAQMEGLGTRLRAQSRQRSRPGRWHKVSVRWEKNRILLVTDGARAWSQEGPHRQHQGAEHPQPHTLFVGGLPASSHSSKLPVTVGFSGCVKRLRLHGRPLGAPTRMAGVTPCILGPLEAGLFFPGSGGVITLDLPGATLPDVGLELEVRPLAVTGLIFHLGQARTPPYLQLQVTEKQVLLRADDGAGEFSTSVTRPSVLCDGQWHRLAVMKSGNVLRLEVDAQSNHTVGPLLAAAAGAPAPLYLGGLPEPMAVQPWPPAYCGCMRRLAVNRSPVAMTRSVEVHGAVGASGCPAA",
# VGF_HUMAN
"MKALRLSASALFCLLLINGLGAAPPGRPEAQPPPLSSEHKEPVAGDAVPGPKDGSAPEVRGARNSEPQDEGELFQGVDPRALAAVLLQALDRPASPPAPSGSQQGPEEEAAEALLTETVRSQTHSLPAPESPEPAAPPRPQTPENGPEASDPSEELEALASLLQELRDFSPSSAKRQQETAAAETETRTHTLTRVNLESPGPERVWRASWGEFQARVPERAPLPPPAPSQFQARMPDSGPLPETHKFGEGVSSPKTHLGEALAPLSKAYQGVAAPFPKARRPESALLGGSEAGERLLQQGLAQVEAGRRQAEATRQAAAQEERLADLASDLLLQYLLQGGARQRGLGGRGLQEAAEERESAREEEEAEQERRGGEERVGEEDEEAAEAEAEAEEAERARQNALLFAEEEDGEAGAEDKRSQEETPGHRRKEAEGTEEGGEEEDDEEMDPQTIDSLIELSTKLHLPADDVVSIIEEVEEKRKRKKNAPPEPVPPPRAAPAPTHVRSPQPPPPAPAPARDELPDWNEVLPPWDREEDEVYPPGPYHPFPNYIRPRTLQPPSALRRRHYHHALPPSRHYPGREAQARRAQEEAEAEERRLQEQEELENYIEHVLLRRP",
# S22A2_HUMAN
"MPTTVDDVLEHGGEFHFFQKQMFFLLALLSATFAPIYVGIVFLGFTPDHRCRSPGVAELSLRCGWSPAEELNYTVPGPGPAGEASPRQCRRYEVDWNQSTFDCVDPLASLDTNRSRLPLGPCRDGWVYETPGSSIVTEFNLVCANSWMLDLFQSSVNVGFFIGSMSIGYIADRFGRKLCLLTTVLINAAAGVLMAISPTYTWMLIFRLIQGLVSKAGWLIGYILITEFVGRRYRRTVGIFYQVAYTVGLLVLAGVAYALPHWRWLQFTVSLPNFFFLLYYWCIPESPRWLISQNKNAEAMRIIKHIAKKNGKSLPASLQRLRLEEETGKKLNPSFLDLVRTPQIRKHTMILMYNWFTSSVLYQGLIMHMGLAGDNIYLDFFYSALVEFPAAFMIILTIDRIGRRYPWAASNMVAGAACLASVFIPGDLQWLKIIISCLGRMGITMAYEIVCLVNAELYPTFIRNLGVHICSSMCDIGGIITPFLVYRLTNIWLELPLMVFGVLGLVAGGLVLLLPETKGKALPETIEEAENMQRPRKNKEKMIYLQVQKLDIPLN",
# OGT1_HUMAN
"MASSVGNVADSTEPTKRMLSFQGLAELAHREYQAGDFEAAERHCMQLWRQEPDNTGVLLLLSSIHFQCRRLDRSAHFSTLAIKQNPLLAEAYSNLGNVYKERGQLQEAIEHYRHALRLKPDFIDGYINLAAALVAAGDMEGAVQAYVSALQYNPDLYCVRSDLGNLLKALGRLEEAKACYLKAIETQPNFAVAWSNLGCVFNAQGEIWLAIHHFEKAVTLDPNFLDAYINLGNVLKEARIFDRAVAAYLRALSLSPNHAVVHGNLACVYYEQGLIDLAIDTYRRAIELQPHFPDAYCNLANALKEKGSVAEAEDCYNTALRLCPTHADSLNNLANIKREQGNIEEAVRLYRKALEVFPEFAAAHSNLASVLQQQGKLQEALMHYKEAIRISPTFADAYSNMGNTLKEMQDVQGALQCYTRAIQINPAFADAHSNLASIHKDSGNIPEAIASYRTALKLKPDFPDAYCNLAHCLQIVCDWTDYDERMKKLVSIVADQLEKNRLPSVHPHHSMLYPLSHGFRKAIAERHGNLCLDKINVLHKPPYEHPKDLKLSDGRLRVGYVSSDFGNHPTSHLMQSIPGMHNPDKFEVFCYALSPDDGTNFRVKVMAEANHFIDLSQIPCNGKAADRIHQDGIHILVNMNGYTKGARNELFALRPAPIQAMWLGYPGTSGALFMDYIITDQETSPAEVAEQYSEKLAYMPHTFFIGDHANMFPHLKKKAVIDFKSNGHIYDNRIVLNGIDLKAFLDSLPDVKIVKMKCPDGGDNADSSNTALNMPVIPMNTIAEAVIEMINRGQIQITINGFSISNGLATTQINNKAATGEEVPRTIIVTTRSQYGLPEDAIVYCNFNQLYKIDPSTLQMWANILKRVPNSVLWLLRFPAVGEPNIQQYAQNMGLPQNRIIFSPVAPKEEHVRRGQLADVCLDTPLCNGHTTGMDVLWAGTPMVTMPGETLASRVAASQLTCLGCLELIAKNRQEYEDIAVKLGTDLEYLKKVRGKVWKQRISSPLFNTKQYTMELERLYLQMWEHYAAGNKPDHMIKPVEVTESA",
# EIF3D_HUMAN
"MAKFMTPVIQDNPSGWGPCAVPEQFRDMPYQPFSKGDRLGKVADWTGATYQDKRYTNKYSSQFGGGSQYAYFHEEDESSFQLVDTARTQKTAYQRNRMRFAQRNLRRDKDRRNMLQFNLQILPKSAKQKERERIRLQKKFQKQFGVRQKWDQKSQKPRDSSVEVRSDWEVKEEMDFPQLMKMRYLEVSEPQDIECCGALEYYDKAFDRITTRSEKPLRSIKRIFHTVTTTDDPVIRKLAKTQGNVFATDAILATLMSCTRSVYSWDIVVQRVGSKLFFDKRDNSDFDLLTVSETANEPPQDEGNSFNSPRNLAMEATYINHNFSQQCLRMGKERYNFPNPNPFVEDDMDKNEIASVAYRYRRWKLGDDIDLIVRCEHDGVMTGANGEVSFINIKTLNEWDSRHCNGVDWRQKLDSQRGAVIATELKNNSYKLARWTCCALLAGSEYLKLGYVSRYHVKDSSRHVILGTQQFKPNEFASQINLSVENAWGILRCVIDICMKLEEGKYLILKDPNKQVIRVYSLPDGTFSSDEDEEEEEEEEEEEEEEET",
# NCAM2_HUMAN
"MSLLLSFYLLGLLVSSGQALLQVTISLSKVELSVGESKFFTCTAIGEPESIDWYNPQGEKIISTQRVVVQKEGVRSRLTIYNANIEDAGIYRCQATDAKGQTQEATVVLEIYQKLTFREVVSPQEFKQGEDAEVVCRVSSSPAPAVSWLYHNEEVTTISDNRFAMLANNNLQILNINKSDEGIYRCEGRVEARGEIDFRDIIVIVNVPPAISMPQKSFNATAERGEEMTFSCRASGSPEPAISWFRNGKLIEENEKYILKGSNTELTVRNIINSDGGPYVCRATNKAGEDEKQAFLQVFVQPHIIQLKNETTYENGQVTLVCDAEGEPIPEITWKRAVDGFTFTEGDKSLDGRIEVKGQHGSSSLHIKDVKLSDSGRYDCEAASRIGGHQKSMYLDIEYAPKFISNQTIYYSWEGNPINISCDVKSNPPASIHWRRDKLVLPAKNTTNLKTYSTGRKMILEIAPTSDNDFGRYNCTATNHIGTRFQEYILALADVPSSPYGVKIIELSQTTAKVSFNKPDSHGGVPIHHYQVDVKEVASEIWKIVRSHGVQTMVVLNNLEPNTTYEIRVAAVNGKGQGDYSKIEIFQTLPVREPSPPSIHGQPSSGKSFKLSITKQDDGGAPILEYIVKYRSKDKEDQWLEKKVQGNKDHIILEHLQWTMGYEVQITAANRLGYSEPTVYEFSMPPKPNIIKDTLFNGLGLGAVIGLGVAALLLILVVTDVSCFFIRQCGLLMCITRRMCGKKSGSSGKSKELEEGKAAYLKDGSKEPIVEMRTEDERVTNHEDGSPVNEPNETTPLTEPEKLPLKEEDGKEALNPETIEIKVSNDIIQSKEDDSKA",
# NMDE4_HUMAN
"MRGAGGPRGPRGPAKMLLLLALACASPFPEEAPGPGGAGGPGGGLGGARPLNVALVFSGPAYAAEAARLGPAVAAAVRSPGLDVRPVALVLNGSDPRSLVLQLCDLLSGLRVHGVVFEDDSRAPAVAPILDFLSAQTSLPIVAVHGGAALVLTPKEKGSTFLQLGSSTEQQLQVIFEVLEEYDWTSFVAVTTRAPGHRAFLSYIEVLTDGSLVGWEHRGALTLDPGAGEAVLSAQLRSVSAQIRLLFCAREEAEPVFRAAEEAGLTGSGYVWFMVGPQLAGGGGSGAPGEPPLLPGGAPLPAGLFAVRSAGWRDDLARRVAAGVAVVARGAQALLRDYGFLPELGHDCRAQNRTHRGESLHRYFMNITWDNRDYSFNEDGFLVNPSLVVISLTRDRTWEVVGSWEQQTLRLKYPLWSRYGRFLQPVDDTQHLTVATLEERPFVIVEPADPISGTCIRDSVPCRSQLNRTHSPPPDAPRPEKRCCKGFCIDILKRLAHTIGFSYDLYLVTNGKHGKKIDGVWNGMIGEVFYQRADMAIGSLTINEERSEIVDFSVPFVETGISVMVARSNGTVSPSAFLEPYSPAVWVMMFVMCLTVVAVTVFIFEYLSPVGYNRSLATGKRPGGSTFTIGKSIWLLWALVFNNSVPVENPRGTTSKIMVLVWAFFAVIFLASYTANLAAFMIQEEYVDTVSGLSDRKFQRPQEQYPPLKFGTVPNGSTEKNIRSNYPDMHSYMVRYNQPRVEEALTQLKAGKLDAFIYDAAVLNYMARKDEGCKLVTIGSGKVFATTGYGIALHKGSRWKRPIDLALLQFLGDDEIEMLERLWLSGICHNDKIEVMSSKLDIDNMAGVFYMLLVAMGLSLLVFAWEHLVYWRLRHCLGPTHRMDFLLAFSRGMYSCCSAEAAPPPAKPPPPPQPLPSPAYPAPRPAPGPAPFVPRERASVDRWRRTKGAGPPGGAGLADGFHRYYGPIEPQGLGLGLGEARAAPRGAAGRPLSPPAAQPPQKPPPSYFAIVRDKEPAEPPAGAFPGFPSPPAPPAAAATAVGPPLCRLAFEDESPPAPARWPRSDPESQPLLGPGAGGAGGTGGAGGGAPAAPPPCRAAPPPCPYLDLEPSPSDSEDSESLGGASLGGLEPWWFADFPYPYAERLGPPPGRYWSVDKLGGWRAGSWDYLPPRSGPAAWHCRHCASLELLPPPRHLSCSHDGLDGGWWAPPPPPWAAGPLPRRRARCGCPRSHPHRPRASHRTPAAAAPHHHRHRRAAGGWDLPPPAPTSRSLEDLSSCPRAAPARRLTGPSRHARRCPHAAHWGPPLPTASHRRHRGGDLGTRRGSAHFSSLESEV",
# STX7_HUMAN
"MSYTPGVGGDPAQLAQRISSNIQKITQCSVEIQRTLNQLGTPQDSPELRQQLQQKQQYTNQLAKETDKYIKEFGSLPTTPSEQRQRKIQKDRLVAEFTTSLTNFQKVQRQAAEREKEFVARVRASSRVSGSFPEDSSKERNLVSWESQTQPQVQVQDEEITEDDLRLIHERESSIRQLEADIMDINEIFKDLGMMIHEQGDVIDSIEANVENAEVHVQQANQQLSRAADYQRKSRKTLCIIILILVIGVAIISLIIWGLNH",
# ARPC5_HUMAN
"MSKNTVSSARFRKVDVDEYDENKFVDEEDGGDGQAGPDEGEVDSCLRQGNMTAALQAALKNPPINTKSQAVKDRAGSIVLKVLISFKANDIEKAVQSLDKNGVDLLMKYIYKGFESPSDNSSAMLLQWHEKALAAGGVGSIVRVLTARKTV",
# P2RX6_HUMAN
"MCPQLAGAGSMGSPGATTGWGLLDYKTEKYVMTRNWRVGALQRLLQFGIVVYVVGWALLAKKGYQERDLEPQFSIITKLKGVSVTQIKELGNRLWDVADFVKPPQGENVFFLVTNFLVTPAQVQGRCPEHPSVPLANCWVDEDCPEGEGGTHSHGVKTGQCVVFNGTHRTCEIWSWCPVESGVVPSRPLLAQAQNFTLFIKNTVTFSKFNFSKSNALETWDPTYFKHCRYEPQFSPYCPVFRIGDLVAKAGGTFEDLALLGGSVGIRVHWDCDLDTGDSGCWPHYSFQLQEKSYNFRTATHWWEQPGVEARTLLKLYGIRFDILVTGQAGKFGLIPTAVTLGTGAAWLGVVTFFCDLLLLYVDREAHFYWRTKYEEAKAPKATANSVWRELALASQARLAECLRRSSAPAPTATAAGSQTQTPGWPCPSSDTHLPTHSGSL",
# FLRT2_HUMAN
"MGLQTTKWPSHGAFFLKSWLIISLGLYSQVSKLLACPSVCRCDRNFVYCNERSLTSVPLGIPEGVTVLYLHNNQINNAGFPAELHNVQSVHTVYLYGNQLDEFPMNLPKNVRVLHLQENNIQTISRAALAQLLKLEELHLDDNSISTVGVEDGAFREAISLKLLFLSKNHLSSVPVGLPVDLQELRVDENRIAVISDMAFQNLTSLERLIVDGNLLTNKGIAEGTFSHLTKLKEFSIVRNSLSHPPPDLPGTHLIRLYLQDNQINHIPLTAFSNLRKLERLDISNNQLRMLTQGVFDNLSNLKQLTARNNPWFCDCSIKWVTEWLKYIPSSLNVRGFMCQGPEQVRGMAVRELNMNLLSCPTTTPGLPLFTPAPSTASPTTQPPTLSIPNPSRSYTPPTPTTSKLPTIPDWDGRERVTPPISERIQLSIHFVNDTSIQVSWLSLFTVMAYKLTWVKMGHSLVGGIVQERIVSGEKQHLSLVNLEPRSTYRICLVPLDAFNYRAVEDTICSEATTHASYLNNGSNTASSHEQTTSHSMGSPFLLAGLIGGAVIFVLVVLLSVFCWHMHKKGRYTSQKWKYNRGRRKDDYCEAGTKKDNSILEMTETSFQIVSLNNDQLLKGDFRLQPIYTPNGGINYTDCHIPNNMRYCNSSVPDLEHCHT",
# PLXB1_HUMAN
"MPALGPALLQALWAGWVLTLQPLPPTAFTPNGTYLQHLARDPTSGTLYLGATNFLFQLSPGLQLEATVSTGPVLDSRDCLPPVMPDECPQAQPTNNPNQLLLVSPGALVVCGSVHQGVCEQRRLGQLEQLLLRPERPGDTQYVAANDPAVSTVGLVAQGLAGEPLLFVGRGYTSRGVGGGIPPITTRALWPPDPQAAFSYEETAKLAVGRLSEYSHHFVSAFARGASAYFLFLRRDLQAQSRAFRAYVSRVCLRDQHYYSYVELPLACEGGRYGLIQAAAVATSREVAHGEVLFAAFSSAAPPTVGRPPSAAAGASGASALCAFPLDEVDRLANRTRDACYTREGRAEDGTEVAYIEYDVNSDCAQLPVDTLDAYPCGSDHTPSPMASRVPLEATPILEWPGIQLTAVAVTMEDGHTIAFLGDSQGQLHRVYLGPGSDGHPYSTQSIQQGSAVSRDLTFDGTFEHLYVMTQSTLLKVPVASCAQHLDCASCLAHRDPYCGWCVLLGRCSRRSECSRGQGPEQWLWSFQPELGCLQVAAMSPANISREETREVFLSVPDLPPLWPGESYSCHFGEHQSPALLTGSGVMCPSPDPSEAPVLPRGADYVSVSVELRFGAVVIAKTSLSFYDCVAVTELRPSAQCQACVSSRWGCNWCVWQHLCTHKASCDAGPMVASHQSPLVSPDPPARGGPSPSPPTAPKALATPAPDTLPVEPGAPSTATASDISPGASPSLLSPWGPWAGSGSISSPGSTGSPLHEEPSPPSPQNGPGTAVPAPTDFRPSATPEDLLASPLSPSEVAAVPPADPGPEALHPTVPLDLPPATVPATTFPGAMGSVKPALDWLTREGGELPEADEWTGGDAPAFSTSTLLSGDGDSAELEGPPAPLILPSSLDYQYDTPGLWELEEATLGASSCPCVESVQGSTLMPVHVEREIRLLGRNLHLFQDGPGDNECVMELEGLEVVVEARVECEPPPDTQCHVTCQQHQLSYEALQPELRVGLFLRRAGRLRVDSAEGLHVVLYDCSVGHGDCSRCQTAMPQYGCVWCEGERPRCVTREACGEAEAVATQCPAPLIHSVEPLTGPVDGGTRVTIRGSNLGQHVQDVLGMVTVAGVPCAVDAQEYEVSSSLVCITGASGEEVAGATAVEVPGRGRGVSEHDFAYQDPKVHSIFPARGPRAGGTRLTLNGSKLLTGRLEDIRVVVGDQPCHLLPEQQSEQLRCETSPRPTPATLPVAVWFGATERRLQRGQFKYTLDPNITSAGPTKSFLSGGREICVRGQNLDVVQTPRIRVTVVSRMLQPSQGLGRRRRVVPETACSLGPSCSSQQFEEPCHVNSSQLITCRTPALPGLPEDPWVRVEFILDNLVFDFATLNPTPFSYEADPTLQPLNPEDPTMPFRHKPGSVFSVEGENLDLAMSKEEVVAMIGDGPCVVKTLTRHHLYCEPPVEQPLPRHHALREAPDSLPEFTVQMGNLRFSLGHVQYDGESPGAFPVAAQVGLGVGTSLLALGVIIIVLMYRRKSKQALRDYKKVQIQLENLESSVRDRCKKEFTDLMTEMTDLTSDLLGSGIPFLDYKVYAERIFFPGHRESPLHRDLGVPESRRPTVEQGLGQLSNLLNSKLFLTKFIHTLESQRTFSARDRAYVASLLTVALHGKLEYFTDILRTLLSDLVAQYVAKNPKLMLRRTETVVEKLLTNWMSICLYTFVRDSVGEPLYMLFRGIKHQVDKGPVDSVTGKAKYTLNDNRLLREDVEYRPLTLNALLAVGPGAGEAQGVPVKVLDCDTISQAKEKMLDQLYKGVPLTQRPDPRTLDVEWRSGVAGHLILSDEDVTSEVQGLWRRLNTLQHYKVPDGATVALVPCLTKHVLRENQDYVPGERTPMLEDVDEGGIRPWHLVKPSDEPEPPRPRRGSLRGGERERAKAIPEIYLTRLLSMKGTLQKFVDDLFQVILSTSRPVPLAVKYFFDLLDEQAQQHGISDQDTIHIWKTNSLPLRFWINIIKNPQFVFDVQTSDNMDAVLLVIAQTFMDACTLADHKLGRDSPINKLLYARDIPRYKRMVERYYADIRQTVPASDQEMNSVLAELSWNYSGDLGARVALHELYKYINKYYDQIITALEEDGTAQKMQLGYRLQQIAAAVENKVTDL",
# PJA2_HUMAN
"MSQYTEKEPAAMDQESGKAVWPKPAGGYQTITGRRYGRRHAYVSFKPCMTRHERSLGRAGDDYEVLELDDVPKENSSGSSPLDQVDSSLPSEPIFEKSETEIPTCGSALNQTTESSQSFVAVHHSEEGRDTLGSSTNLHNHSEGEYIPGACSASSVQNGIALVHTDSYDPDGKHGEDNDHLQLSAEVVEGSRYQESLGNTVFELENREAEAYTGLSPPVPSFNCEVRDEFEELDSVPLVKSSAGDTEFVHQNSQEIQRSSQDEMVSTKQQNNTSQERQTEHSPEDAACGPGHICSEQNTNDREKNHGSSPEQVVRPKVRKLISSSQVDQETGFNRHEAKQRSVQRWREALEVEESGSDDLLIKCEEYDGEHDCMFLDPPYSRVITQRETENNQMTSESGATAGRQEVDNTFWNGCGDYYQLYDKDEDSSECSDGEWSASLPHRFSGTEKDQSSSDESWETLPGKDENEPELQSDSSGPEEENQELSLQEGEQTSLEEGEIPWLQYNEVNESSSDEGNEPANEFAQPAFMLDGNNNLEDDSSVSEDLDVDWSLFDGFADGLGVAEAISYVDPQFLTYMALEERLAQAMETALAHLESLAVDVEVANPPASKESIDGLPETLVLEDHTAIGQEQCCPICCSEYIKDDIATELPCHHFFHKPCVSIWLQKSGTCPVCRRHFPPAVIEASAAPSSEPDPDAPPSNDSIAEAP",
# SI1L1_HUMAN
"MTSLKRSQTERPLATDRASVVGTDGTPKVHTDDFYMRRFRSQNGSLGSSVMAPVGPPRSEGSHHITSTPGVPKMGVRARIADWPPRKENIKESSRSSQEIETSSCLDSLSSKSSPVSQGSSVSLNSNDSAMLKSIQNTLKNKTRPSENMDSRFLMPEAYPSSPRKALRRIRQRSNSDITISELDVDSFDECISPTYKTGPSLHREYGSTSSIDKQGTSGESFFDLLKGYKDDKSDRGPTPTKLSDFLITGGGKGSGFSLDVIDGPISQRENLRLFKEREKPLKRRSKSETGDSSIFRKLRNAKGEELGKSSDLEDNRSEDSVRPWTCPKCFAHYDVQSILFDLNEAIMNRHNVIKRRNTTTGASAAAVASLVSGPLSHSASFSSPMGSTEDLNSKGSLSMDQGDDKSNELVMSCPYFRNEIGGEGERKISLSKSNSGSFSGCESASFESTLSSHCTNAGVAVLEVPKENLVLHLDRVKRYIVEHVDLGAYYYRKFFYQKEHWNYFGADENLGPVAVSIRREKPDEMKENGSPYNYRIIFRTSELMTLRGSVLEDAIPSTAKHSTARGLPLKEVLEHVVPELNVQCLRLAFNTPKVTEQLMKLDEQGLNYQQKVGIMYCKAGQSTEEEMYNNESAGPAFEEFLQLLGERVRLKGFEKYRAQLDTKTDSTGTHSLYTTYKDYEIMFHVSTMLPYTPNNKQQLLRKRHIGNDIVTIVFQEPGAQPFSPKNIRSHFQHVFVIVRVHNPCSDSVCYSVAVTRSRDVPSFGPPIPKGVTFPKSNVFRDFLLAKVINAENAAHKSEKFRAMATRTRQEYLKDLAEKNVTNTPIDPSGKFPFISLASKKKEKSKPYPGAELSSMGAIVWAVRAEDYNKAMELDCLLGISNEFIVLIEQETKSVVFNCSCRDVIGWTSTDTSLKIFYERGECVSVGSFINIEEIKEIVKRLQFVSKGCESVEMTLRRNGLGQLGFHVNYEGIVADVEPYGYAWQAGLRQGSRLVEICKVAVATLSHEQMIDLLRTSVTVKVVIIPPHDDCTPRRSCSETYRMPVMEYKMNEGVSYEFKFPFRNNNKWQRNASKGPHSPQVPSQVQSPMTSRLNAGKGDGKMPPPERAANIPRSISSDGRPLERRLSPGSDIYVTVSSMALARSQCRNSPSNLSSSSDTGSVGGTYRQKSMPEGFGVSRRSPASIDRQNTQSDIGGSGKSTPSWQRSEDSIADQMAYSYRGPQDFNSFVLEQHEYTEPTCHLPAVSKVLPAFRESPSGRLMRQDPVVHLSPNKQGHSDSHYSSHSSSNTLSSNASSAHSDEKWYDGDRTESELNSYNYLQGTSADSGIDTTSYGPSHGSTASLGAATSSPRSGPGKEKVAPLWHSSSEVISMADRTLETESHGLDRKTESSLSLDIHSKSQAGSTPLTRENSTFSINDAASHTSTMSSRHSASPVVFTSARSSPKEELHPAAPSQLAPSFSSSSSSSSGPRSFYPRQGATSKYLIGWKKPEGTINSVGFMDTRKRHQSDGNEIAHTRLRASTRDLRASPKPTSKSTIEEDLKKLIDLESPTPESQKSFKFHALSSPQSPFPSTPTSRRALHRTLSDESIYNSQREHFFTSRASLLDQALPNDVLFSSTYPSLPKSLPLRRPSYTLGMKSLHGEFSASDSSLTDIQETRRQPMPDPGLMPLPDTAADLDWSNLVDAAKAYEVQRASFFAASDENHRPLSAASNSDQLEDQALAQMKPYSSSKDSSPTLASKVDQLEGMLKMLREDLKKEKEDKAHLQAEVQHLREDNLRLQEESQNASDKLKKFTEWVFNTIDMS",
# SEPT4_HUMAN
"MDRSLGWQGNSVPEDRTEAGIKRFLEDTTDDGELSKFVKDFSGNASCHPPEAKTWASRPQVPEPRPQAPDLYDDDLEFRPPSRPQSSDNQQYFCAPAPLSPSARPRSPWGKLDPYDSSEDDKEYVGFATLPNQVHRKSVKKGFDFTLMVAGESGLGKSTLVNSLFLTDLYRDRKLLGAEERIMQTVEITKHAVDIEEKGVRLRLTIVDTPGFGDAVNNTECWKPVAEYIDQQFEQYFRDESGLNRKNIQDNRVHCCLYFISPFGHGLRPLDVEFMKALHQRVNIVPILAKADTLTPPEVDHKKRKIREEIEHFGIKIYQFPDCDSDEDEDFKLQDQALKESIPFAVIGSNTVVEARGRRVRGRLYPWGIVEVENPGHCDFVKLRTMLVRTHMQDLKDVTRETHYENYRAQCIQSMTRLVVKERNRNKLTRESGTDFPIPAVPPGTDPETEKLIREKDEELRRMQEMLHKIQKQMKENY",
# SRGP3_HUMAN
"MSSQTKFKKDKEIIAEYEAQIKEIRTQLVEQFKCLEQQSESRLQLLQDLQEFFRRKAEIELEYSRSLEKLAERFSSKIRSSREHQFKKDQYLLSPVNCWYLVLHQTRRESRDHATLNDIFMNNVIVRLSQISEDVIRLFKKSKEIGLQMHEELLKVTNELYTVMKTYHMYHAESISAESKLKEAEKQEEKQFNKSGDLSMNLLRHEDRPQRRSSVKKIEKMKEKRQAKYSENKLKCTKARNDYLLNLAATNAAISKYYIHDVSDLIDCCDLGFHASLARTFRTYLSAEYNLETSRHEGLDVIENAVDNLDSRSDKHTVMDMCNQVFCPPLKFEFQPHMGDEVCQVSAQQPVQTELLMRYHQLQSRLATLKIENEEVRKTLDATMQTLQDMLTVEDFDVSDAFQHSRSTESVKSAASETYMSKINIAKRRANQQETEMFYFTKFKEYVNGSNLITKLQAKHDLLKQTLGEGERAECGTTRPPCLPPKPQKMRRPRPLSVYSHKLFNGSMEAFIKDSGQAIPLVVESCIRYINLYGLQQQGIFRVPGSQVEVNDIKNSFERGEDPLVDDQNERDINSVAGVLKLYFRGLENPLFPKERFQDLISTIKLENPAERVHQIQQILVTLPRVVIVVMRYLFAFLNHLSQYSDENMMDPYNLAICFGPTLMHIPDGQDPVSCQAHINEVIKTIIIHHEAIFPSPRELEGPVYEKCMAGGEEYCDSPHSEPGAIDEVDHDNGTEPHTSDEEVEQIEAIAKFDYMGRSPRELSFKKGASLLLYHRASEDWWEGRHNGVDGLIPHQYIVVQDMDDAFSDSLSQKADSEASSGPLLDDKASSKNDLQSPTEHISDYGFGGVMGRVRLRSDGAAIPRRRSGGDTHSPPRGLGPSIDTPPRAAACPSSPHKIPLTRGRIESPEKRRMATFGSAGSINYPDKKALSEGHSMRSTCGSTRHSSLGDHKSLEAEALAEDIEKTMSTALHELRELERQNTVKQAPDVVLDTLEPLKNPPGPVSSEPASPLHTIVIRDPDAAMRRSSSSSTEMMTTFKPALSARLAGAQLRPPPMRPVRPVVQHRSSSSSSSGVGSPAVTPTEKMFPNSSADKSGTM",
# LRRT2_HUMAN
"MGLHFKWPLGAPMLAAIYAMSMVLKMLPALGMACPPKCRCEKLLFYCDSQGFHSVPNATDKGSLGLSLRHNHITELERDQFASFSQLTWLHLDHNQISTVKEDAFQGLYKLKELILSSNKIFYLPNTTFTQLINLQNLDLSFNQLSSLHPELFYGLRKLQTLHLRSNSLRTIPVRLFWDCRSLEFLDLSTNRLRSLARNGFAGLIKLRELHLEHNQLTKINFAHFLRLSSLHTLFLQWNKISNLTCGMEWTWGTLEKLDLTGNEIKAIDLTVFETMPNLKILLMDNNKLNSLDSKILNSLRSLTTVGLSGNLWECSARICALASWLGSFQGRWEHSILCHSPDHTQGEDILDAVHGFQLCWNLSTTVTVMATTYRDPTTEYTKRISSSSYHVGDKEIPTTAGIAVTTEEHFPEPDNAIFTQRVITGTMALLFSFFFIIFIVFISRKCCPPTLRRIRQCSMVQNHRQLRSQTRLHMSNMSDQGPYNEYEPTHEGPFIIINGYGQCKCQQLPYKECEV",
# ARHG9_HUMAN
"MTLLITGDSIVSAEAVWDHVTMANRELAFKAGDVIKVLDASNKDWWWGQIDDEEGWFPASFVRLWVNQEDEVEEGPSDVQNGHLDPNSDCLCLGRPLQNRDQMRANVINEIMSTERHYIKHLKDICEGYLKQCRKRRDMFSDEQLKVIFGNIEDIYRFQMGFVRDLEKQYNNDDPHLSEIGPCFLEHQDGFWIYSEYCNNHLDACMELSKLMKDSRYQHFFEACRLLQQMIDIAIDGFLLTPVQKICKYPLQLAELLKYTAQDHSDYRYVAAALAVMRNVTQQINERKRRLENIDKIAQWQASVLDWEGEDILDRSSELIYTGEMAWIYQPYGRNQQRVFFLFDHQMVLCKKDLIRRDILYYKGRIDMDKYEVVDIEDGRDDDFNVSMKNAFKLHNKETEEIHLFFAKKLEEKIRWLRAFREERKMVQEDEKIGFEISENQKRQAAMTVRKVPKQKGVNSARSVPPSYPPPQDPLNHGQYLVPDGIAQSQVFEFTEPKRSQSPFWQNFSRLTPFKK",
# GRID2_HUMAN
"MEVFPFLLVLSVWWSRTWDSANADSIIHIGAIFDESAKKDDEVFRTAVGDLNQNEEILQTEKITFSVTFVDGNNPFQAVQEACELMNQGILALVSSIGCTSAGSLQSLADAMHIPHLFIQRSTAGTPRSGCGLTRSNRNDDYTLSVRPPVYLHDVILRVVTEYAWQKFIIFYDSEYDIRGIQEFLDKVSQQGMDVALQKVENNINKMITTLFDTMRIEELNRYRDTLRRAILVMNPATAKSFITEVVETNLVAFDCHWIIINEEINDVDVQELVRRSIGRLTIIRQTFPVPQNISQRCFRGNHRISSTLCDPKDPFAQNMEISNLYIYDTVLLLANAFHKKLEDRKWHSMASLSCIRKNSKPWQGGRSMLETIKKGGVSGLTGELEFGENGGNPNVHFEILGTNYGEELGRGVRKLGCWNPVTGLNGSLTDKKLENNMRGVVLRVVTVLEEPFVMVSENVLGKPKKYQGFSIDVLDALSNYLGFNYEIYVAPDHKYGSPQEDGTWNGLVGELVFKRADIGISALTITPDRENVVDFTTRYMDYSVGVLLRRAEKTVDMFACLAPFDLSLWACIAGTVLLVGLLVYLLNWLNPPRLQMGSMTSTTLYNSMWFVYGSFVQQGGEVPYTTLATRMMMGAWWLFALIVISSYTANLAAFLTITRIESSIQSLQDLSKQTEIPYGTVLDSAVYEHVRMKGLNPFERDSMYSQMWRMINRSNGSENNVLESQAGIQKVKYGNYAFVWDAAVLEYVAINDPDCSFYTIGNTVADRGYGIALQHGSPYRDVFSQRILELQQNGDMDILKHKWWPKNGQCDLYSSVDTKQKGGALDIKSFAGVFCILAAGIVLSCFIAMLETWWNKRKGSRVPSKEDDKEIDLEHLHRRVNSLCTDDDSPHKQFSTSSIDLTPLDIDTLPTRQALEQISDFRNTHITTTTFIPEQIQTLSRTLSAKAASGFTFGNVPEHRTGPFRHRAPNGGFFRSPIKTMSSIPYQPTPTLGLNLGNDPDRGTSI",
# SYNJ1_HUMAN
"MAFSKGFRIYHKLDPPPFSLIVETRHKEECLMFESGAVAVLSSAEKEAIKGTYSKVLDAYGLLGVLRLNLGDTMLHYLVLVTGCMSVGKIQESEVFRVTSTEFISLRIDSSDEDRISEVRKVLNSGNFYFAWSASGISLDLSLNAHRSMQEQTTDNRFFWNQSLHLHLKHYGVNCDDWLLRLMCGGVEIRTIYAAHKQAKACLISRLSCERAGTRFNVRGTNDDGHVANFVETEQVVYLDDSVSSFIQIRGSVPLFWEQPGLQVGSHRVRMSRGFEANAPAFDRHFRTLKNLYGKQIIVNLLGSKEGEHMLSKAFQSHLKASEHAADIQMVNFDYHQMVKGGKAEKLHSVLKPQVQKFLDYGFFYFNGSEVQRCQSGTVRTNCLDCLDRTNSVQAFLGLEMLAKQLEALGLAEKPQLVTRFQEVFRSMWSVNGDSISKIYAGTGALEGKAKLKDGARSVTRTIQNNFFDSSKQEAIDVLLLGNTLNSDLADKARALLTTGSLRVSEQTLQSASSKVLKSMCENFYKYSKPKKIRVCVGTWNVNGGKQFRSIAFKNQTLTDWLLDAPKLAGIQEFQDKRSKPTDIFAIGFEEMVELNAGNIVSASTTNQKLWAVELQKTISRDNKYVLLASEQLVGVCLFVFIRPQHAPFIRDVAVDTVKTGMGGATGNKGAVAIRMLFHTTSLCFVCSHFAAGQSQVKERNEDFIEIARKLSFPMGRMLFSHDYVFWCGDFNYRIDLPNEEVKELIRQQNWDSLIAGDQLINQKNAGQVFRGFLEGKVTFAPTYKYDLFSDDYDTSEKCRTPAWTDRVLWRRRKWPFDRSAEDLDLLNASFQDESKILYTWTPGTLLHYGRAELKTSDHRPVVALIDIDIFEVEAEERQNIYKEVIAVQGPPDGTVLVSIKSSLPENNFFDDALIDELLQQFASFGEVILIRFVEDKMWVTFLEGSSALNVLSLNGKELLNRTITIALKSPDWIKNLEEEMSLEKISIALPSSTSSTLLGEDAEVAADFDMEGDVDDYSAEVEELLPQHLQPSSSSGLGTSPSSSPRTSPCQSPTISEGPVPSLPIRPSRAPSRTPGPPSAQSSPIDAQPATPLPQKDPAQPLEPKRPPPPRPVAPPTRPAPPQRPPPPSGARSPAPTRKEFGGIGAPPSPGVARREMEAPKSPGTTRKDNIGRSQPSPQAGLAGPGPAGYSTARPTIPPRAGVISAPQSHARASAGRLTPESQSKTSETSKGSTFLPEPLKPQAAFPPQSSLPPPAQRLQEPLVPVAAPMPQSGPQPNLETPPQPPPRSRSSHSLPSEASSQPQVKTNGISDGKRESPLKIDPFEDLSFNLLAVSKAQLSVQTSPVPTPDPKRLIQLPSATQSNVLSSVSCMPTMPPIPARSQSQENMRSSPNPFITGLTRTNPFSDRTAAPGNPFRAKSEESEATSWFSKEEPVTISPFPSLQPLGHNKSRASSSLDGFKDSFDLQGQSTLKISNPKGWVTFEEEEDFGVKGKSKSACSDLLGNQPSSFSGSNLTLNDDWNKGTNVSFCVLPSRRPPPPPVPLLPPGTSPPVDPFTTLASKASPTLDFTER",
# IF4G3_HUMAN
"MNSQPQTRSPFFQRPQIQPPRATIPNSSPSIRPGAQTPTAVYQANQHIMMVNHLPMPYPVPQGPQYCIPQYRHSGPPYVGPPQQYPVQPPGPGPFYPGPGPGDFPNAYGTPFYPSQPVYQSAPIIVPTQQQPPPAKREKKTIRIRDPNQGGKDITEEIMSGGGSRNPTPPIGRPTSTPTPPQQLPSQVPEHSPVVYGTVESAHLAASTPVTAASDQKQEEKPKPDPVLKSPSPVLRLVLSGEKKEQEGQTSETTAIVSIAELPLPPSPTTVSSVARSTIAAPTSSALSSQPIFTTAIDDRCELSSPREDTIPIPSLTSCTETSDPLPTNENDDDICKKPCSVAPNDIPLVSSTNLINEINGVSEKLSATESIVEIVKQEVLPLTLELEILENPPEEMKLECIPAPITPSTVPSFPPTPPTPPASPPHTPVIVPAAATTVSSPSAAITVQRVLEEDESIRTCLSEDAKEIQNKIEVEADGQTEEILDSQNLNSRRSPVPAQIAITVPKTWKKPKDRTRTTEEMLEAELELKAEEELSIDKVLESEQDKMSQGFHPERDPSDLKKVKAVEENGEEAEPVRNGAESVSEGEGIDANSGSTDSSGDGVTFPFKPESWKPTDTEGKKQYDREFLLDFQFMPACIQKPEGLPPISDVVLDKINQPKLPMRTLDPRILPRGPDFTPAFADFGRQTPGGRGVPLLNVGSRRSQPGQRREPRKIITVSVKEDVHLKKAENAWKPSQKRDSQADDPENIKTQELFRKVRSILNKLTPQMFNQLMKQVSGLTVDTEERLKGVIDLVFEKAIDEPSFSVAYANMCRCLVTLKVPMADKPGNTVNFRKLLLNRCQKEFEKDKADDDVFEKKQKELEAASAPEERTRLHDELEEAKDKARRRSIGNIKFIGELFKLKMLTEAIMHDCVVKLLKNHDEESLECLCRLLTTIGKDLDFEKAKPRMDQYFNQMEKIVKERKTSSRIRFMLQDVIDLRLCNWVSRRADQGPKTIEQIHKEAKIEEQEEQRKVQQLMTKEKRRPGVQRVDEGGWNTVQGAKNSRVLDPSKFLKITKPTIDEKIQLVPKAQLGSWGKGSSGGAKASETDALRSSASSLNRFSALQPPAPSGSTPSTPVEFDSRRTLTSRGSMGREKNDKPLPSATARPNTFMRGGSSKDLLDNQSQEEQRREMLETVKQLTGGVDVERNSTEAERNKTRESAKPEISAMSAHDKAALSEEELERKSKSIIDEFLHINDFKEAMQCVEELNAQGLLHVFVRVGVESTLERSQITRDHMGQLLYQLVQSEKLSKQDFFKGFSETLELADDMAIDIPHIWLYLAELVTPMLKEGGISMRELTIEFSKPLLPVGRAGVLLSEILHLLCKQMSHKKVGALWREADLSWKDFLPEGEDVHNFLLEQKLDFIESDSPCSSEALSKKELSAEELYKRLEKLIIEDKANDEQIFDWVEANLDEIQMSSPTFLRALMTAVCKAAIIADSSTFRVDTAVIKQRVPILLKYLDSDTEKELQALYALQASIVKLDQPANLLRMFFDCLYDEEVISEDAFYKWESSKDPAEQNGKGVALKSVTAFFTWLREAEEESEDN",
# TGON2_HUMAN
"MRFVVALVLLNVAAAGAVPLLATESVKQEEAGVRPSAGNVSTHPSLSQRPGGSTKSHPEPQTPKDSPSKSSAEAQTPEDTPNKSGAEAKTQKDSSNKSGAEAKTQKGSTSKSGSEAQTTKDSTSKSHPELQTPKDSTGKSGAEAQTPEDSPNRSGAEAKTQKDSPSKSGSEAQTTKDVPNKSGADGQTPKDGSSKSGAEDQTPKDVPNKSGAEKQTPKDGSNKSGAEEQGPIDGPSKSGAEEQTSKDSPNKVVPEQPSRKDHSKPISNPSDNKELPKADTNQLADKGKLSPHAFKTESGEETDLISPPQEEVKSSEPTEDVEPKEAEDDDTGPEEGSPPKEEKEKMSGSASSENREGTLSDSTGSEKDDLYPNGSGNGSAESSHFFAYLVTAAILVAVLYIAHHNKRKIIAFVLEGKRSKVTRRPKASDYQRLDQKS",
# KCNQ3_HUMAN
"MGLKARRAAGAAGGGGDGGGGGGGAANPAGGDAAAAGDEERKVGLAPGDVEQVTLALGAGADKDGTLLLEGGGRDEGQRRTPQGIGLLAKTPLSRPVKRNNAKYRRIQTLIYDALERPRGWALLYHALVFLIVLGCLILAVLTTFKEYETVSGDWLLLLETFAIFIFGAEFALRIWAAGCCCRYKGWRGRLKFARKPLCMLDIFVLIASVPVVAVGNQGNVLATSLRSLRFLQILRMLRMDRRGGTWKLLGSAICAHSKELITAWYIGFLTLILSSFLVYLVEKDVPEVDAQGEEMKEEFETYADALWWGLITLATIGYGDKTPKTWEGRLIAATFSLIGVSFFALPAGILGSGLALKVQEQHRQKHFEKRRKPAAELIQAAWRYYATNPNRIDLVATWRFYESVVSFPFFRKEQLEAASSQKLGLLDRVRLSNPRGSNTKGKLFTPLNVDAIEESPSKEPKPVGLNNKERFRTAFRMKAYAFWQSSEDAGTGDPMAEDRGYGNDFPIEDMIPTLKAAIRAVRILQFRLYKKKFKETLRPYDVKDVIEQYSAGHLDMLSRIKYLQTRIDMIFTPGPPSTPKHKKSQKGSAFTFPSQQSPRNEPYVARPSTSEIEDQSMMGKFVKVERQVQDMGKKLDFLVDMHMQHMERLQVQVTEYYPTKGTSSPAEAEKKEDNRYSDLKTIICNYSETGPPEPPYSFHQVTIDKVSPYGFFAHDPVNLPRGGPSSGKVQATPPSSATTYVERPTVLPILTLLDSRVSCHSQADLQGPYSDRISPRQRRSITRDSDTPLSLMSVNHEELERSPSGFSISQDRDDYVFGPNGGSSWMREKRYLAEGETDTDTDPFTPSGSMPLSSTGDGISDSVWTPSNKPI",
# KCNQ2_HUMAN
"MVQKSRNGGVYPGPSGEKKLKVGFVGLDPGAPDSTRDGALLIAGSEAPKRGSILSKPRAGGAGAGKPPKRNAFYRKLQNFLYNVLERPRGWAFIYHAYVFLLVFSCLVLSVFSTIKEYEKSSEGALYILEIVTIVVFGVEYFVRIWAAGCCCRYRGWRGRLKFARKPFCVIDIMVLIASIAVLAAGSQGNVFATSALRSLRFLQILRMIRMDRRGGTWKLLGSVVYAHSKELVTAWYIGFLCLILASFLVYLAEKGENDHFDTYADALWWGLITLTTIGYGDKYPQTWNGRLLAATFTLIGVSFFALPAGILGSGFALKVQEQHRQKHFEKRRNPAAGLIQSAWRFYATNLSRTDLHSTWQYYERTVTVPMYSSQTQTYGASRLIPPLNQLELLRNLKSKSGLAFRKDPPPEPSPSKGSPCRGPLCGCCPGRSSQKVSLKDRVFSSPRGVAAKGKGSPQAQTVRRSPSADQSLEDSPSKVPKSWSFGDRSRARQAFRIKGAASRQNSEEASLPGEDIVDDKSCPCEFVTEDLTPGLKVSIRAVCVMRFLVSKRKFKESLRPYDVMDVIEQYSAGHLDMLSRIKSLQSRVDQIVGRGPAITDKDRTKGPAEAELPEDPSMMGRLGKVEKQVLSMEKKLDFLVNIYMQRMGIPPTETEAYFGAKEPEPAPPYHSPEDSREHVDRHGCIVKIVRSSSSTGQKNFSAPPAAPPVQCPPSTSWQPQSHPRQGHGTSPVGDHGSLVRIPPPPAHERSLSAYGGGNRASMEFLRQEDTPGCRPPEGNLRDSDTSISIPSVDHEELERSFSGFSISQSKENLDALNSCYAAVAPCAKVRPYIAEGESDTDSDLCTPCGPPPRSATGEGPFGDVGWAGPRK",
# RGS14_HUMAN
"MPGKPKHLGVPNGRMVLAVSDGELSSTTGPQGQGEGRGSSLSIHSLPSGPSSPFPTEEQPVASWALSFERLLQDPLGLAYFTEFLKKEFSAENVTFWKACERFQQIPASDTQQLAQEARNIYQEFLSSQALSPVNIDRQAWLGEEVLAEPRPDMFRAQQLQIFNLMKFDSYARFVKSPLYRECLLAEAEGRPLREPGSSRLGSPDATRKKPKLKPGKSLPLGVEELGQLPPVEGPGGRPLRKSFRRELGGTANAALRRESQGSLNSSASLDLGFLAFVSSKSESHRKSLGSTEGESESRPGKYCCVYLPDGTASLALARPGLTIRDMLAGICEKRGLSLPDIKVYLVGNEQALVLDQDCTVLADQEVRLENRITFELELTALERVVRISAKPTKRLQEALQPILEKHGLSPLEVVLHRPGEKQPLDLGKLVSSVAAQRLVLDTLPGVKISKARDKSPCRSQGCPPRTQDKATHPPPASPSSLVKVPSSATGKRQTCDIEGLVELLNRVQSSGAHDQRGLLRKEDLVLPEFLQLPAQGPSSEETPPQTKSAAQPIGGSLNSTTDSAL",
# SYT7_HUMAN
"MYRDPEAASPGAPSRDVLLVSAIITVSLSVTVVLCGLCHWCQRKLGKRYKNSLETVGTPDSGRGRSEKKAIKLPAGGKAVNTAPVPGQTPHDESDRRTEPRSSVSDLVNSLTSEMLMLSPGSEEDEAHEGCSRENLGRIQFSVGYNFQESTLTVKIMKAQELPAKDFSGTSDPFVKIYLLPDKKHKLETKVKRKNLNPHWNETFLFEGFPYEKVVQRILYLQVLDYDRFSRNDPIGEVSIPLNKVDLTQMQTFWKDLKPCSDGSGSRGELLLSLCYNPSANSIIVNIIKARNLKAMDIGGTSDPYVKVWLMYKDKRVEKKKTVTMKRNLNPIFNESFAFDIPTEKLRETTIIITVMDKDKLSRNDVIGKIYLSWKSGPGEVKHWKDMIARPRQPVAQWHQLKA",
# DCX_HUMAN
"MELDFGHFDERDKTSRNMRGSRMNGLPSPTHSAHCSFYRTRTLQALSNEKKAKKVRFYRNGDRYFKGIVYAVSSDRFRSFDALLADLTRSLSDNINLPQGVRYIYTIDGSRKIGSMDELEEGESYVCSSDNFFKKVEYTKNVNPNWSVNVKTSANMKAPQSLASSNSAQARENKDFVRPKLVTIIRSGVKPRKAVRVLLNKKTAHSFEQVLTDITEAIKLETGVVKKLYTLDGKQVTCLHDFFGDDDVFIACGPEKFRYAQDDFSLDENECRVMKGNPSATAGPKASPTPQKTSAKSPGPMRRSKSPADSGNDQDANGTSSSQLSTPKSKQSPISTPTSPGSLRKHKDLYLPLSLDDSDSLGDSM",
# OREX_HUMAN
"MNLPSTKVSWAAVTLLLLLLLLPPALLSSGAAAQPLPDCCRQKTCSCRLYELLHGAGNHAAGILTLGKRRSGPPGLQGRLQRLLQASGNHAAGILTMGRRAGAEPAPRPCLGRRCSAPAAASVAPGGQSGI",
# AKA7A_HUMAN
"MGQLCCFPFSRDEGKISELESSSSAVLQRYSKDIPSWSSGEKNGGEPDDAELVRLSKRLVENAVLKAVQQYLEETQNKNKPGEGSSVKTEAADQNGNDNENNRK",
# ACTN4_HUMAN
"MVDYHAANQSYQYGPSSAGNGAGGGGSMGDYMAQEDDWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIENIDEDFRDGLKLMLLLEVISGERLPKPERGKMRVHKINNVNKALDFIASKGVKLVSIGAEEIVDGNAKMTLGMIWTIILRFAIQDISVEETSAKEGLLLWCQRKTAPYKNVNVQNFHISWKDGLAFNALIHRHRPELIEYDKLRKDDPVTNLNNAFEVAEKYLDIPKMLDAEDIVNTARPDEKAIMTYVSSFYHAFSGAQKAETAANRICKVLAVNQENEHLMEDYEKLASDLLEWIRRTIPWLEDRVPQKTIQEMQQKLEDFRDYRRVHKPPKVQEKCQLEINFNTLQTKLRLSNRPAFMPSEGKMVSDINNGWQHLEQAEKGYEEWLLNEIRRLERLDHLAEKFRQKASIHEAWTDGKEAMLKHRDYETATLSDIKALIRKHEAFESDLAAHQDRVEQIAAIAQELNELDYYDSHNVNTRCQKICDQWDALGSLTHSRREALEKTEKQLEAIDQLHLEYAKRAAPFNNWMESAMEDLQDMFIVHTIEEIEGLISAHDQFKSTLPDADREREAILAIHKEAQRIAESNHIKLSGSNPYTTVTPQIINSKWEKVQQLVPKRDHALLEEQSKQQSNEHLRRQFASQANVVGPWIQTKMEEIGRISIEMNGTLEDQLSHLKQYERSIVDYKPNLDLLEQQHQLIQEALIFDNKHTNYTMEHIRVGWEQLLTTIARTINEVENQILTRDAKGISQEQMQEFRASFNHFDKDHGGALGPEEFKACLISLGYDVENDRQGEAEFNRIMSLVDPNHSGLVTFQAFIDFMSRETTDTDTADQVIASFKVLAGDKNFITAEELRRELPPDQAEYCIARMAPYQGPDAVPGALDYKSFSTALYGESDL",
# AP1G1_HUMAN
"MPAPIRLRELIRTIRTARTQAEEREMIQKECAAIRSSFREEDNTYRCRNVAKLLYMHMLGYPAHFGQLECLKLIASQKFTDKRIGYLGAMLLLDERQDVHLLMTNCIKNDLNHSTQFVQGLALCTLGCMGSSEMCRDLAGEVEKLLKTSNSYLRKKAALCAVHVIRKVPELMEMFLPATKNLLNEKNHGVLHTSVVLLTEMCERSPDMLAHFRKLVPQLVRILKNLIMSGYSPEHDVSGISDPFLQVRILRLLRILGRNDDDSSEAMNDILAQVATNTETSKNVGNAILYETVLTIMDIKSESGLRVLAINILGRFLLNNDKNIRYVALTSLLKTVQTDHNAVQRHRSTIVDCLKDLDVSIKRRAMELSFALVNGNNIRGMMKELLYFLDSCEPEFKADCASGIFLAAEKYAPSKRWHIDTIMRVLTTAGSYVRDDAVPNLIQLITNSVEMHAYTVQRLYKAILGDYSQQPLVQVAAWCIGEYGDLLVSGQCEEEEPIQVTEDEVLDILESVLISNMSTSVTRGYALTAIMKLSTRFTCTVNRIKKVVSIYGSSIDVELQQRAVEYNALFKKYDHMRSALLERMPVMEKVTTNGPTEIVQTNGETEPAPLETKPPPSGPQPTSQANDLLDLLGGNDITPVIPTAPTSKPSSAGGELLDLLGDINLTGAPAAAPAPASVPQISQPPFLLDGLSSQPLFNDIAAGIPSITAYSKNGLKIEFTFERSNTNPSVTVITIQASNSTELDMTDFVFQAAVPKTFQLQLLSPSSSIVPAFNTGTITQVIKVLNPQKQQLRMRIKLTYNHKGSAMQDLAEVNNFPPQSWQ",
# STX6_HUMAN
"MSMEDPFFVVKGEVQKAVNTAQGLFQRWTELLQDPSTATREEIDWTTNELRNNLRSIEWDLEDLDETISIVEANPRKFNLDATELSIRKAFITSTRQVVRDMKDQMSTSSVQALAERKNRQALLGDSGSQNWSTGTTDKYGRLDRELQRANSHFIEEQQAQQQLIVEQQDEQLELVSGSIGVLKNMSQRIGGELEEQAVMLEDFSHELESTQSRLDNVMKKLAKVSHMTSDRRQWCAIAILFAVLLVVLILFLVL",
# SNG1_HUMAN
"MEGGAYGAGKAGGAFDPYTLVRQPHTILRVVSWLFSIVVFGSIVNEGYLNSASEGEEFCIYNRNPNACSYGVAVGVLAFLTCLLYLALDVYFPQISSVKDRKKAVLSDIGVSAFWAFLWFVGFCYLANQWQVSKPKDNPLNEGTDAARAAIAFSFFSIFTWAGQAVLAFQRYQIGADSALFSQDYMDPSQDSSMPYAPYVEPTGPDPAGMGGTYQQPANTFDTEPQGYQSQGY",
# SNG3_HUMAN
"MEGASFGAGRAGAALDPVSFARRPQTLLRVASWVFSIAVFGPIVNEGYVNTDSGPELRCVFNGNAGACRFGVALGLGAFLACAAFLLLDVRFQQISSVRDRRRAVLLDLGFSGLWSFLWFVGFCFLTNQWQRTAPGPATTQAGDAARAAIAFSFFSILSWVALTVKALQRFRLGTDMSLFATEQLSTGASQAYPGYPVGSGVEGTETYQSPPFTETLDTSPKGYQVPAY",
# SGTA_HUMAN
"MDNKKRLAYAIIQFLHDQLRHGGLSSDAQESLEVAIQCLETAFGVTVEDSDLALPQTLPEIFEAAATGKEMPQDLRSPARTPPSEEDSAEAERLKTEGNEQMKVENFEAAVHFYGKAIELNPANAVYFCNRAAAYSKLGNYAGAVQDCERAICIDPAYSKAYGRMGLALSSLNKHVEAVAYYKKALELDPDNETYKSNLKIAELKLREAPSPTGGVGSFDIAGLLNNPGFMSMASNLMNNPQIQQLMSGMISGGNNPLGTPGTSPSQNDLASLIQAGQQFAQQMQQQNPELIEQLRSQIRSRTPSASNDDQQE",
# STRN_HUMAN
"MDEQAGPGVFFSNNHPGAGGAKGLGPLAEAAAAGDGAAAAGAARAQYSLPGILHFLQHEWARFEVERAQWEVERAELQAQIAFLQGERKGQENLKKDLVRRIKMLEYALKQERAKYHKLKYGTELNQGDMKPPSYDSDEGNETEVQPQQNSQLMWKQGRQLLRQYLQEVGYTDTILDVKSKRVRALLGFSSDVTDREDDKNQDSVVNGTEAEVKETAMIAKSELTDSASVLDNFKFLESAAADFSDEDEDDDVDGREKSVIDTSTIVRKKALPDSGEDRDTKEALKEFDFLVTSEEGDNESRSAGDGTDWEKEDQCLMPEAWNVDQGVITKLKEQYKKERKGKKGVKRPNRSKLQDMLANLRDVDELPSLQPSVGSPSRPSSSRLPEHEINRADEVEALTFPPSSGKSFIMGADEALESELGLGELAGLTVANEADSLTYDIANNKDALRKTWNPKFTLRSHFDGIRALAFHPIEPVLITASEDHTLKMWNLQKTAPAKKSTSLDVEPIYTFRAHKGPVLCVVMSSNGEQCYSGGTDGLIQGWNTTNPNIDPYDSYDPSVLRGPLLGHTDAVWGLAYSAAHQRLLSCSADGTLRLWNTTEVAPALSVFNDTKELGIPASVDLVSSDPSHMVASFSKGYTSIFNMETQQRILTLESNVDTTANSSCQINRVISHPTLPISITAHEDRHIKFYDNNTGKLIHSMVAHLEAVTSLAVDPNGLYLMSGSHDCSIRLWNLESKTCIQEFTAHRKKFEESIHDVAFHPSKCYIASAGADALAKVFV",
# KALRN_HUMAN
"MTDRFWDQWYLWYLRLLRLLDRGSFRNDGLKASDVLPILKEKVAFVSGGRDKRGGPILTFPARSNHDRIRQEDLRKLVTYLASVPSEDVCKRGFTVIIDMRGSKWDLIKPLLKTLQEAFPAEIHVALIIKPDNFWQKQKTNFGSSKFIFETSMVSVEGLTKLVDPSQLTEEFDGSLDYNHEEWIELRLSLEEFFNSAVHLLSRLEDLQEMLARKEFPVDVEGSRRLIDEHTQLKKKVLKAPVEELDREGQRLLQCIRCSDGFSGRNCIPGSADFQSLVPKITSLLDKLHSTRQHLHQMWHVRKLKLDQCFQLRLFEQDAEKMFDWISHNKELFLQSHTEIGVSYQYALDLQTQHNHFAMNSMNAYVNINRIMSVASRLSEAGHYASQQIKQISTQLDQEWKSFAAALDERSTILAMSAVFHQKAEQFLSGVDAWCKMCSEGGLPSEMQDLELAIHHHQTLYEQVTQAYTEVSQDGKALLDVLQRPLSPGNSESLTATANYSKAVHQVLDVVHEVLHHQRRLESIWQHRKVRLHQRLQLCVFQQDVQQVLDWIENHGEAFLSKHTGVGKSLHRARALQKRHDDFEEVAQNTYTNADKLLEAAEQLAQTGECDPEEIYKAARHLEVRIQDFVRRVEQRKLLLDMSVSFHTHTKELWTWMEDLQKEMLEDVCADSVDAVQELIKQFQQQQTATLDATLNVIKEGEDLIQQLRSAPPSLGEPSEARDSAVSNNKTPHSSSISHIESVLQQLDDAQVQMEELFHERKIKLDIFLQLRIFEQYTIEVTAELDAWNEDLLRQMNDFNTEDLTLAEQRLQRHTERKLAMNNMTFEVIQQGQDLHQYITEVQASGIELICEKDIDLAAQVQELLEFLHEKQHELELNAEQTHKRLEQCLQLRHLQAEVKQVLGWIRNGESMLNASLVNASSLSEAEQLQREHEQFQLAIESLFHATSLQKTHQSALQVQQKAEVLLQAGHYDADAIRECAEKVALHWQQLMLKMEDRLKLVNASVAFYKTSEQVCSVLESLEQEYRRDEDWCGGRDKLGPAAEIDHVIPLISKHLEQKEAFLKACTLARRNAEVFLKYIHRNNVSMPSVASHTRGPEQQVKAILSELLQRENRVLHFWTLKKRRLDQCQQYVVFERSAKQALDWIQETGEFYLSTHTSTGETTEETQELLKEYGEFRVPAKQTKEKVKLLIQLADSFVEKGHIHATEIRKWVTTVDKHYRDFSLRMGKYRYSLEKALGVNTEDNKDLELDIIPASLSDREVKLRDANHEVNEEKRKSARKKEFIMAELLQTEKAYVRDLHECLETYLWEMTSGVEEIPPGILNKEHIIFGNIQEIYDFHNNIFLKELEKYEQLPEDVGHCFVTWADKFQMYVTYCKNKPDSNQLILEHAGTFFDEIQQRHGLANSISSYLIKPVQRITKYQLLLKELLTCCEEGKGELKDGLEVMLSVPKKANDAMHVSMLEGFDENLDVQGELILQDAFQVWDPKSLIRKGRERHLFLFEISLVFSKEIKDSSGHTKYVYKNKLLTSELGVTEHVEGDPCKFALWSGRTPSSDNKTVLKASNIETKQEWIKNIREVIQERIIHLKGALKEPLQLPKTPAKQRNNSKRDGVEDIDSQGDGSSQPDTISIASRTSQNTVDSDKLSGGCELTVVLQDFSAGHSSELTIQVGQTVELLERPSERPGWCLVRTTERSPPLEGLVPSSALCISHSRSSVEMDCFFPLVKDAYSHSSSENGGKSESVANLQAQPSLNSIHSSPGPKRSTNTLKKWLTSPVRRLNSGKADGNIKKQKKVRDGRKSFDLGSPKPGDETTPQGDSADEKSKKGWGEDEPDEESHTPLPPPMKIFDNDPTQDEMSSSLLAARQASTEVPTAADLVNAIEKLVKNKLSLEGSSYRGSLKDPAGCLNEGMAPPTPPKNPEEEQKAKALRGRMFVLNELVQTEKDYVKDLGIVVEGFMKRIEEKGVPEDMRGKDKIVFGNIHQIYDWHKDFFLAELEKCIQEQDRLAQLFIKHERKLHIYVWYCQNKPRSEYIVAEYDAYFEEVKQEINQRLTLSDFLIKPIQRITKYQLLLKDFLRYSEKAGLECSDIEKAVELMCLVPKRCNDMMNLGRLQGFEGTLTAQGKLLQQDTFYVIELDAGMQSRTKERRVFLFEQIVIFSELLRKGSLTPGYMFKRSIKMNYLVLEENVDNDPCKFALMNRETSERVVLQAANADIQQAWVQDINQVLETQRDFLNALQSPIEYQRKERSTAVMRSQPARLPQASPRPYSSVPAGSEKPPKGSSYNPPLPPLKISTSNGSPGFEYHQPGDKFEASKQNDLGGCNGTSSMAVIKDYYALKENEICVSQGEVVQVLAVNQQNMCLVYQPASDHSPAAEGWVPGSILAPLTKATAAESSDGSIKKSCSWHTLRMRKRAEVENTGKNEATGPRKPKDILGNKVSVKETNSSEESECDDLDPNTSMEILNPNFIQEVAPEFLVPLVDVTCLLGDTVILQCKVCGRPKPTITWKGPDQNILDTDNSSATYTVSSCDSGEITLKICNLMPQDSGIYTCIATNDHGTTSTSATVKVQGVPAAPNRPIAQERSCTSVILRWLPPSSTGNCTISGYTVEYREEGSQIWQQSVASTLDTYLVIEDLSPGCPYQFRVSASNPWGISLPSEPSEFVRLPEYDAAADGATISWKENFDSAYTELNEIGRGRFSIVKKCIHKATRKDVAVKFVSKKMKKKEQAAHEAALLQHLQHPQYITLHDTYESPTSYILILELMDDGRLLDYLMNHDELMEEKVAFYIRDIMEALQYLHNCRVAHLDIKPENLLIDLRIPVPRVKLIDLEDAVQISGHFHIHHLLGNPEFAAPEVIQGIPVSLGTDIWSIGVLTYVMLSGVSPFLDESKEETCINVCRVDFSFPHEYFCGVSNAARDFINVILQEDFRRRPTAATCLQHPWLQPHNGSYSKIPLDTSRLACFIERRKHQNDVRPIPNVKSYIVNRVNQGT",
# AGRB2_HUMAN
"MENTGWMGKGHRMTPACPLLLSVILSLRLATAFDPAPSACSALASGVLYGAFSLQDLFPTIASGCSWTLENPDPTKYSLYLRFNRQEQVCAHFAPRLLPLDHYLVNFTCLRPSPEEAVAQAESEVGRPEEEEAEAAAGLELCSGSGPFTFLHFDKNFVQLCLSAEPSEAPRLLAPAALAFRFVEVLLINNNNSSQFTCGVLCRWSEECGRAAGRACGFAQPGCSCPGEAGAGSTTTTSPGPPAAHTLSNALVPGGPAPPAEADLHSGSSNDLFTTEMRYGEEPEEEPKVKTQWPRSADEPGLYMAQTGDPAAEEWSPWSVCSLTCGQGLQVRTRSCVSSPYGTLCSGPLRETRPCNNSATCPVHGVWEEWGSWSLCSRSCGRGSRSRMRTCVPPQHGGKACEGPELQTKLCSMAACPVEGQWLEWGPWGPCSTSCANGTQQRSRKCSVAGPAWATCTGALTDTRECSNLECPATDSKWGPWNAWSLCSKTCDTGWQRRFRMCQATGTQGYPCEGTGEEVKPCSEKRCPAFHEMCRDEYVMLMTWKKAAAGEIIYNKCPPNASGSASRRCLLSAQGVAYWGLPSFARCISHEYRYLYLSLREHLAKGQRMLAGEGMSQVVRSLQELLARRTYYSGDLLFSVDILRNVTDTFKRATYVPSADDVQRFFQVVSFMVDAENKEKWDDAQQVSPGSVHLLRVVEDFIHLVGDALKAFQSSLIVTDNLVISIQREPVSAVSSDITFPMRGRRGMKDWVRHSEDRLFLPKEVLSLSSPGKPATSGAAGSPGRGRGPGTVPPGPGHSHQRLLPADPDESSYFVIGAVLYRTLGLILPPPRPPLAVTSRVMTVTVRPPTQPPAEPLITVELSYIINGTTDPHCASWDYSRADASSGDWDTENCQTLETQAAHTRCQCQHLSTFAVLAQPPKDLTLELAGSPSVPLVIGCAVSCMALLTLLAIYAAFWRFIKSERSIILLNFCLSILASNILILVGQSRVLSKGVCTMTAAFLHFFFLSSFCWVLTEAWQSYLAVIGRMRTRLVRKRFLCLGWGLPALVVAVSVGFTRTKGYGTSSYCWLSLEGGLLYAFVGPAAVIVLVNMLIGIIVFNKLMARDGISDKSKKQRAGSERCPWASLLLPCSACGAVPSPLLSSASARNAMASLWSSCVVLPLLALTWMSAVLAMTDRRSVLFQALFAVFNSAQGFVITAVHCFLRREVQDVVKCQMGVCRADESEDSPDSCKNGQLQILSDFEKDVDLACQTVLFKEVNTCNPSTITGTLSRLSLDEDEEPKSCLVGPEGSLSFSPLPGNILVPMAASPGLGEPPPPQEANPVYMCGEGGLRQLDLTWLRPTEPGSEGDYMVLPRRTLSLQPGGGGGGGEDAPRARPEGTPRRAAKTVAHTEGYPSFLSVDHSGLGLGPAYGSLQNPYGMTFQPPPPTPSARQVPEPGERSRTMPRTVPGSTMKMGSLERKKLRYSDLDFEKVMHTRKRHSELYHELNQKFHTFDRYRSQSTAKREKRWSVSSGGAAERSVCTDKPSPGERPSLSQHRRHQSWSTFKSMTLGSLPPKPRERLTLHRAAAWEPTEPPDGDFQTEV",
# AGRB3_HUMAN
"MKAVRNLLIYIFSTYLLVMFGFNAAQDFWCSTLVKGVIYGSYSVSEMFPKNFTNCTWTLENPDPTKYSIYLKFSKKDLSCSNFSLLAYQFDHFSHEKIKDLLRKNHSIMQLCNSKNAFVFLQYDKNFIQIRRVFPTNFPGLQKKGEEDQKSFFEFLVLNKVSPSQFGCHVLCTWLESCLKSENGRTESCGIMYTKCTCPQHLGEWGIDDQSLILLNNVVLPLNEQTEGCLTQELQTTQVCNLTREAKRPPKEEFGMMGDHTIKSQRPRSVHEKRVPQEQADAAKFMAQTGESGVEEWSQWSTCSVTCGQGSQVRTRTCVSPYGTHCSGPLRESRVCNNTALCPVHGVWEEWSPWSLCSFTCGRGQRTRTRSCTPPQYGGRPCEGPETHHKPCNIALCPVDGQWQEWSSWSQCSVTCSNGTQQRSRQCTAAAHGGSECRGPWAESRECYNPECTANGQWNQWGHWSGCSKSCDGGWERRIRTCQGAVITGQQCEGTGEEVRRCNEQRCPAPYEICPEDYLMSMVWKRTPAGDLAFNQCPLNATGTTSRRCSLSLHGVAFWEQPSFARCISNEYRHLQHSIKEHLAKGQRMLAGDGMSQVTKTLLDLTQRKNFYAGDLLMSVEILRNVTDTFKRASYIPASDGVQNFFQIVSNLLDEENKEKWEDAQQIYPGSIELMQVIEDFIHIVGMGMMDFQNSYLMTGNVVASIQKLPAASVLTDINFPMKGRKGMVDWARNSEDRVVIPKSIFTPVSSKELDESSVFVLGAVLYKNLDLILPTLRNYTVINSKIIVVTIRPEPKTTDSFLEIELAHLANGTLNPYCVLWDDSKTNESLGTWSTQGCKTVLTDASHTKCLCDRLSTFAILAQQPREIIMESSGTPSVTLIVGSGLSCLALITLAVVYAALWRYIRSERSIILINFCLSIISSNILILVGQTQTHNKSICTTTTAFLHFFFLASFCWVLTEAWQSYMAVTGKIRTRLIRKRFLCLGWGLPALVVATSVGFTRTKGYGTDHYCWLSLEGGLLYAFVGPAAAVVLVNMVIGILVFNKLVSRDGILDKKLKHRAGQMSEPHSGLTLKCAKCGVVSTTALSATTASNAMASLWSSCVVLPLLALTWMSAVLAMTDKRSILFQILFAVFDSLQGFVIVMVHCILRREVQDAFRCRLRNCQDPINADSSSSFPNGHAQIMTDFEKDVDIACRSVLHKDIGPCRAATITGTLSRISLNDDEEEKGTNPEGLSYSTLPGNVISKVIIQQPTGLHMPMSMNELSNPCLKKENSELRRTVYLCTDDNLRGADMDIVHPQERMMESDYIVMPRSSVNNQPSMKEESKMNIGMETLPHERLLHYKVNPEFNMNPPVMDQFNMNLEQHLAPQEHMQNLPFEPRTAVKNFMASELDDNAGLSRSETGSTISMSSLERRKSRYSDLDFEKVMHTRKRHMELFQELNQKFQTLDRFRDIPNTSSMENPAPNKNPWDTFKNPSEYPHYTTINVLDTEAKDALELRPAEWEKCLNLPLDVQEGDFQTEV",
# PCDH7_HUMAN
"MLRMRTAGWARGWCLGCCLLLPLSLSLAAAKQLLRYRLAEEGPADVRIGNVASDLGIVTGSGEVTFSLESGSEYLKIDNLTGELSTSERRIDREKLPQCQMIFDENECFLDFEVSVIGPSQSWVDLFEGQVIVLDINDNTPTFPSPVLTLTVEENRPVGTLYLLPTATDRDFGRNGIERYELLQEPGGGGSGGESRRAGAADSAPYPGGGGNGASGGGSGGSKRRLDASEGGGGTNPGGRSSVFELQVADTPDGEKQPQLIVKGALDREQRDSYELTLRVRDGGDPPRSSQAILRVLITDVNDNSPRFEKSVYEADLAENSAPGTPILQLRAADLDVGVNGQIEYVFGAATESVRRLLRLDETSGWLSVLHRIDREEVNQLRFTVMARDRGQPPKTDKATVVLNIKDENDNVPSIEIRKIGRIPLKDGVANVAEDVLVDTPIALVQVSDRDQGENGVVTCTVVGDVPFQLKPASDTEGDQNKKKYFLHTSTPLDYEATREFNVVIVAVDSGSPSLSSNNSLIVKVGDTNDNPPMFGQSVVEVYFPENNIPGERVATVLATDADSGKNAEIAYSLDSSVMGIFAIDPDSGDILVNTVLDREQTDRYEFKVNAKDKGIPVLQGSTTVIVQVADKNDNDPKFMQDVFTFYVKENLQPNSPVGMVTVMDADKGRNAEMSLYIEENNNIFSIENDTGTIYSTMSFDREHQTTYTFRVKAVDGGDPPRSATATVSLFVMDENDNAPTVTLPKNISYTLLPPSSNVRTVVATVLATDSDDGINADLNYSIVGGNPFKLFEIDPTSGVVSLVGKLTQKHYGLHRLVVQVNDSGQPSQSTTTLVHVFVNESVSNATAIDSQIARSLHIPLTQDIAGDPSYEISKQRLSIVIGVVAGIMTVILIILIVVMARYCRSKNKNGYEAGKKDHEDFFTPQQHDKSKKPKKDKKNKKSKQPLYSSIVTVEASKPNGQRYDSVNEKLSDSPSMGRYRSVNGGPGSPDLARHYKSSSPLPTVQLHPQSPTAGKKHQAVQDLPPANTFVGAGDNISIGSDHCSEYSCQTNNKYSKQMRLHPYITVFG",
# PRKN_HUMAN
"MIVFVRFNSSHGFPVEVDSDTSIFQLKEVVAKRQGVPADQLRVIFAGKELRNDWTVQNCDLDQQSIVHIVQRPWRKGQEMNATGGDDPRNAAGGCEREPQSLTRVDLSSSVLPGDSVGLAVILHTDSRKDSPPAGSPAGRSIYNSFYVYCKGPCQRVQPGKLRVQCSTCRQATLTLTQGPSCWDDVLIPNRMSGECQSPHCPGTSAEFFFKCGAHPTSDKETSVALHLIATNSRNITCITCTDVRSPVLVFQCNSRHVICLDCFHLYCVTRLNDRQFVHDPQLGYSLPCVAGCPNSLIKELHHFRILGEEQYNRYQQYGAEECVLQMGGVLCPRPGCGAGLLPEPDQRKVTCEGGNGLGCGFAFCRECKEAYHEGECSAVFEASGTTTQAYRVDERAAEQARWEAASKETIKKTTKPCPRCHVPVEKNGGCMHMKCPQPQCRLEWCWNCGCEWNRVCMGDHWFDV",
# GBG7_HUMAN
"MSATNNIAQARKLVEQLRIEAGIERIKVSKAASDLMSYCEQHARNDPLLVGVPASENPFKDKKPCIIL",
# KIF5C_HUMAN
"MADPAECSIKVMCRFRPLNEAEILRGDKFIPKFKGDETVVIGQGKPYVFDRVLPPNTTQEQVYNACAKQIVKDVLEGYNGTIFAYGQTSSGKTHTMEGKLHDPQLMGIIPRIAHDIFDHIYSMDENLEFHIKVSYFEIYLDKIRDLLDVSKTNLAVHEDKNRVPYVKGCTERFVSSPEEVMDVIDEGKANRHVAVTNMNEHSSRSHSIFLINIKQENVETEKKLSGKLYLVDLAGSEKVSKTGAEGAVLDEAKNINKSLSALGNVISALAEGTKTHVPYRDSKMTRILQDSLGGNCRTTIVICCSPSVFNEAETKSTLMFGQRAKTIKNTVSVNLELTAEEWKKKYEKEKEKNKTLKNVIQHLEMELNRWRNGEAVPEDEQISAKDQKNLEPCDNTPIIDNIAPVVAGISTEEKEKYDEEISSLYRQLDDKDDEINQQSQLAEKLKQQMLDQDELLASTRRDYEKIQEELTRLQIENEAAKDEVKEVLQALEELAVNYDQKSQEVEDKTRANEQLTDELAQKTTTLTTTQRELSQLQELSNHQKKRATEILNLLLKDLGEIGGIIGTNDVKTLADVNGVIEEEFTMARLYISKMKSEVKSLVNRSKQLESAQMDSNRKMNASERELAACQLLISQHEAKIKSLTDYMQNMEQKRRQLEESQDSLSEELAKLRAQEKMHEVSFQDKEKEHLTRLQDAEEMKKALEQQMESHREAHQKQLSRLRDEIEEKQKIIDEIRDLNQKLQLEQEKLSSDYNKLKIEDQEREMKLEKLLLLNDKREQAREDLKGLEETVSRELQTLHNLRKLFVQDLTTRVKKSVELDNDDGGGSAAQKQKISFLENNLEQLTKVHKQLVRDNADLRCELPKLEKRLRATAERVKALESALKEAKENAMRDRKRYQQEVDRIKEAVRAKNMARRAHSAQIAKPIRPGHYPASSPTAVHAIRGGGGSSSNSTHYQK",
# SI1L3_HUMAN
"MTTYRAIPSDGVDLAASCGARVGDVLPGPHTGDYAPLGFWAQNGSMSQPLGESPATATATATATTRPSPTTPAMPKMGVRARVADWPPKREALREHSNPSPSQDTDGTKATKMAHSMRSIQNGQPPTSTPASSGSKAFHRLSRRRSKDVEFQDGWPRSPGRAFLPLRHRSSSEITLSECDAEDAGEPRGARHTGALPLFREYGSTSSIDVQGMPEQSFFDILNEFRSEQPDARGCQALTELLRADPGPHLMGGGGGAKGDSHNGQPAKDSLLPLQPTKEKEKARKKPARGLGGGDTVDSSIFRKLRSSKPEGEAGRSPGEADEGRSPPEASRPWVCQKSFAHFDVQSMLFDLNEAAANRVSVSQRRNTTTGASAASAASAMASLTASRAHSLGGLDPAFTSTEDLNCKENLEQDLGDDNSNDLLLSCPHFRNEIGGECERNVSFSRASVGSPSSGEGHLAEPALSAYRTNASISVLEVPKEQQRTQSRPRQYSIEHVDLGARYYQDYFVGKEHANYFGVDEKLGPVAVSIKREKLEDHKEHGPQYQYRIIFRTRELITLRGSILEDATPTATKHGTGRGLPLKDALEYVIPELNIHCLRLALNTPKVTEQLLKLDEQGLCRKHKVGILYCKAGQSSEEEMYNNEEAGPAFEEFLSLIGEKVCLKGFTKYAAQLDVKTDSTGTHSLYTMYQDYEIMFHVSTLLPYTPNNRQQLLRKRHIGNDIVTIIFQEPGALPFTPKNIRSHFQHVFIIVRVHNPCTDNVCYSMAVTRSKDAPPFGPPIPSGTTFRKSDVFRDFLLAKVINAENAAHKSDKFHTMATRTRQEYLKDLAENCVSNTPIDSTGKFNLISLTSKKKEKTKARAGAEQHSAGAIAWRVVAQDYAQGVEIDCILGISNEFVVLLDLRTKEVVFNCYCGDVIGWTPDSSTLKIFYGRGDHIFLQATEGSVEDIREIVQRLKVMTSGWETVDMTLRRNGLGQLGFHVKYDGTVAEVEDYGFAWQAGLRQGSRLVEICKVAVVTLTHDQMIDLLRTSVTVKVVIIPPFEDGTPRRGWPETYDMNTSEPKTEQESITPGGRPPYRSNAPWQWSGPASHNSLPASKWATPTTPGHAQSLSRPLKQTPIVPFRESQPLHSKRPVSFPETPYTVSPAGADRVPPYRQPSGSFSTPGSATYVRYKPSPERYTAAPHPLLSLDPHFSHDGTSSGDSSSGGLTSQESTMERQKPEPLWHVPAQARLSAIAGSSGNKHPSRQDAAGKDSPNRHSKGEPQYSSHSSSNTLSSNASSSHSDDRWFDPLDPLEPEQDPLSKGGSSDSGIDTTLYTSSPSCMSLAKAPRPAKPHKPPGSMGLCGGGREAAGRSHHADRRREVSPAPAVAGQSKGYRPKLYSSGSSTPTGLAGGSRDPPRQPSDMGSRVGYPAQVYKTASAETPRPSQLAQPSPFQLSASVPKSFFSKQPVRNKHPTGWKRTEEPPPRPLPFSDPKKQVDTNTKNVFGQPRLRASLRDLRSPRKNYKSTIEDDLKKLIIMDNLGPEQERDTGQSPQKGLQRTLSDESLCSGRREPSFASPAGLEPGLPSDVLFTSTCAFPSSTLPARRQHQHPHPPVGPGATPAAGSGFPEKKSTISASELSLADGRDRPLRRLDPGLMPLPDTAAGLEWSSLVNAAKAYEVQRAVSLFSLNDPALSPDIPPAHSPVHSHLSLERGPPTPRTTPTMSEEPPLDLTGKVYQLEVMLKQLHTDLQKEKQDKVVLQSEVASLRQNNQRLQEESQAASEQLRKFAEIFCREKKEL",
# LZTS3_HUMAN
"MAKLETLPVRADPGRDPLLAFAPRPSELGPPDPRLAMGSVGSGVAHAQEFAMKSVGTRTGGGGSQGSFPGPRGSGSGASRERPGRYPSEDKGLANSLYLNGELRGSDHTDVCGNVVGSSGGSSSSGGSDKAPPQYREPSHPPKLLATSGKLDQCSEPLVRPSAFKPVVPKNFHSMQNLCPPQTNGTPEGRQGPGGLKGGLDKSRTMTPAGGSGSGLSDSGRNSLTSLPTYSSSYSQHLAPLSASTSHINRIGTASYGSGSGGSSGGGSGYQDLGTSDSGRASSKSGSSSSMGRPGHLGSGEGGGGGLPFAACSPPSPSALIQELEERLWEKEQEVAALRRSLEQSEAAVAQVLEERQKAWERELAELRQGCSGKLQQVARRAQRAQQGLQLQVLRLQQDKKQLQEEAARLMRQREELEDKVAACQKEQADFLPRIEETKWEVCQKAGEISLLKQQLKDSQADVSQKLSEIVGLRSQLREGRASLREKEEQLLSLRDSFSSKQASLELGEGELPAACLKPALTPVDPAEPQDALATCESDEAKMRRQAGVAAAASLVSVDGEAEAGGESGTRALRREVGRLQAELAAERRARERQGASFAEERRVWLEEKEKVIEYQKQLQLSYVEMYQRNQQLERRLRERGAAGGASTPTPQHGEEKKAWTPSRLERIESTEI",
# PI51C_HUMAN
"MELEVPDEAESAEAGAVPSEAAWAAESGAAAGLAQKKAAPTEVLSMTAQPGPGHGKKLGHRGVDASGETTYKKTTSSTLKGAIQLGIGYTVGHLSSKPERDVLMQDFYVVESIFFPSEGSNLTPAHHFQDFRFKTYAPVAFRYFRELFGIRPDDYLYSLCNEPLIELSNPGASGSLFYVTSDDEFIIKTVMHKEAEFLQKLLPGYYMNLNQNPRTLLPKFYGLYCVQSGGKNIRVVVMNNILPRVVKMHLKFDLKGSTYKRRASKKEKEKSFPTYKDLDFMQDMPEGLLLDADTFSALVKTLQRDCLVLESFKIMDYSLLLGVHNIDQHERERQAQGAQSTSDEKRPVGQKALYSTAMESIQGGAARGEAIESDDTMGGIPAVNGRGERLLLHIGIIDILQSYRFIKKLEHTWKALVHDGDTVSVHRPSFYAERFFKFMSNTVFRKNSSLKSSPSKKGRGGALLAVKPLGPTAAFSASQIPSEREEAQYDLRGARSYPTLEDEGRPDLLPCTPPSFEEATTASIATTLSSTSLSIPERSPSETSEQPRYRRRTQSSGQDGRPQEEPPAEEDLQQITVQVEPACSVEIVVPKEEDAGVEASPAGASAAVEVETASQASDEEGAPASQASDEEDAPATDIYFPTDERSWVYSPLHYSAQAPPASDGESDT",
# KIF1B_HUMAN
"MSGASVKVAVRVRPFNSRETSKESKCIIQMQGNSTSIINPKNPKEAPKSFSFDYSYWSHTSPEDPCFASQNRVYNDIGKEMLLHAFEGYNVCIFAYGQTGAGKSYTMMGKQEESQAGIIPQLCEELFEKINDNCNEEMSYSVEVSYMEIYCERVRDLLNPKNKGNLRVREHPLLGPYVEDLSKLAVTSYTDIADLMDAGNKARTVAATNMNETSSRSHAVFTIVFTQKKHDNETNLSTEKVSKISLVDLAGSERADSTGAKGTRLKEGANINKSLTTLGKVISALAEVDNCTSKSKKKKKTDFIPYRDSVLTWLLRENLGGNSRTAMVAALSPADINYDETLSTLRYADRAKQIKCNAVINEDPNAKLVRELKEEVTRLKDLLRAQGLGDIIDIDPLIDDYSGSGSKYLKDFQNNKHRYLLASENQRPGHFSTASMGSLTSSPSSCSLSSQVGLTSVTSIQERIMSTPGGEEAIERLKESEKIIAELNETWEEKLRKTEAIRMEREALLAEMGVAIREDGGTLGVFSPKKTPHLVNLNEDPLMSECLLYYIKDGITRVGQADAERRQDIVLSGAHIKEEHCIFRSERSNSGEVIVTLEPCERSETYVNGKRVSQPVQLRSGNRIIMGKNHVFRFNHPEQARAEREKTPSAETPSEPVDWTFAQRELLEKQGIDMKQEMEKRLQEMEILYKKEKEEADLLLEQQRLDYESKLQALQKQVETRSLAAETTEEEEEEEEVPWTQHEFELAQWAFRKWKSHQFTSLRDLLWGNAVYLKEANAISVELKKKVQFQFVLLTDTLYSPLPPELLPTEMEKTHEDRPFPRTVVAVEVQDLKNGATHYWSLEKLKQRLDLMREMYDRAGEMASSAQDESETTVTGSDPFYDRFHWFKLVGSSPIFHGCVNERLADRTPSPTFSTADSDITELADEQQDEMEDFDDEAFVDDAGSDAGTEEGSDLFSDGHDPFYDRSPWFILVGRAFVYLSNLLYPVPLIHRVAIVSEKGEVRGFLRVAVQAIAADEEAPDYGSGIRQSGTAKISFDNEYFNQSDFSSVAMTRSGLSLEELRIVEGQGQSSEVITPPEEISRINDLDLKSSTLLDGKMVMEGFSEEIGNHLKLGSAFTFRVTVLQASGILPEYADIFCQFNFLHRHDEAFSTEPLKNNGRGSPLAFYHVQNIAVEITESFVDYIKTKPIVFEVFGHYQQHPLHLQGQELNSPPQPCRRFFPPPMPLSKPVPATKLNTMSKTSLGQSMSKYDLLVWFEISELEPTGEYIPAVVDHTAGLPCQGTFLLHQGIQRRITVTIIHEKGSELHWKDVRELVVGRIRNKPEVDEAAVDAILSLNIISAKYLKSSHNSSRTFYRFEAVWDSSLHNSLLLNRVTPYGEKIYMTLSAYLELDHCIQPAVITKDVCMVFYSRDAKISPPRSLRSLFGSGYSKSPDSNRVTGIYELSLCKMSDTGSPGMQRRRRKILDTSVAYVRGEENLAGWRPRGDSLILEHQWELEKLELLHEVEKTRHFLLLRERLGDSIPKSLSDSLSPSLSSGTLSTSTSISSQISTTTFESAITPSESSGYDSGDIESLVDREKELATKCLQLLTHTFNREFSQVHGSVSDCKLSDISPIGRDPSESSFSSATLTPSSTCPSLVDSRSNSLDQKTPEANSRASSPCPEFEQFQIVPAVETPYLARAGKNEFLNLVPDIEEIRPSSVVSKKGYLHFKEPLYSNWAKHFVVVRRPYVFIYNSDKDPVERGIINLSTAQVEYSEDQQAMVKTPNTFAVCTKHRGVLLQALNDKDMNDWLYAFNPLLAGTIRSKLSRRCPSQSKY",
# CCG3_HUMAN
"MRMCDRGIQMLITTVGAFAAFSLMTIAVGTDYWLYSRGVCRTKSTSDNETSRKNEEVMTHSGLWRTCCLEGAFRGVCKKIDHFPEDADYEQDTAEYLLRAVRASSVFPILSVTLLFFGGLCVAASEFHRSRHNVILSAGIFFVSAGLSNIIGIIVYISANAGDPGQRDSKKSYSYGWSFYFGAFSFIIAEIVGVVAVHIYIEKHQQLRAKSHSEFLKKSTFARLPPYRYRFRRRSSSRSTEPRSRDLSPISKGFHTIPSTDISMFTLSRDPSKITMGTLLNSDRDHAFLQFHNSTPKEFKESLHNNPANRRTTPV",
# NMD3B_HUMAN
"MEFVRALWLGLALALGPGSAGGHPQPCGVLARLGGSVRLGALLPRAPLARARARAALARAALAPRLPHNLSLELVVAAPPARDPASLTRGLCQALVPPGVAALLAFPEARPELLQLHFLAAATETPVLSLLRREARAPLGAPNPFHLQLHWASPLETLLDVLVAVLQAHAWEDVGLALCRTQDPGGLVALWTSRAGRPPQLVLDLSRRDTGDAGLRARLAPMAAPVGGEAPVPAAVLLGCDIARARRVLEAVPPGPHWLLGTPLPPKALPTAGLPPGLLALGEVARPPLEAAIHDIVQLVARALGSAAQVQPKRALLPAPVNCGDLQPAGPESPGRFLARFLANTSFQGRTGPVWVTGSSQVHMSRHFKVWSLRRDPRGAPAWATVGSWRDGQLDLEPGGASARPPPPQGAQVWPKLRVVTLLEHPFVFARDPDEDGQCPAGQLCLDPGTNDSATLDALFAALANGSAPRALRKCCYGYCIDLLERLAEDTPFDFELYLVGDGKYGALRDGRWTGLVGDLLAGRAHMAVTSFSINSARSQVVDFTSPFFSTSLGIMVRARDTASPIGAFMWPLHWSTWLGVFAALHLTALFLTVYEWRSPYGLTPRGRNRSTVFSYSSALNLCYAILFRRTVSSKTPKCPTGRLLMNLWAIFCLLVLSSYTANLAAVMVGDKTFEELSGIHDPKLHHPAQGFRFGTVWESSAEAYIKKSFPDMHAHMRRHSAPTTPRGVAMLTSDPPKLNAFIMDKSLLDYEVSIDADCKLLTVGKPFAIEGYGIGLPQNSPLTSNLSEFISRYKSSGFIDLLHDKWYKMVPCGKRVFAVTETLQMSIYHFAGLFVLLCLGLGSALLSSLGEHAFFRLALPRIRKGSRLQYWLHTSQKIHRALNTEPPEGSKEETAEAEPSGPEVEQQQQQQDQPTAPEGWKRARRAVDKERRVRFLLEPAVVVAPEADAEAEAAPREGPVWLCSYGRPPAARPTGAPQPGELQELERRIEVARERLRQALVRRGQLLAQLGDSARHRPRRLLQARAAPAEAPPHSGRPGSQE",
# EVI5_HUMAN
"MVTNKMTAAFRNPSGKQVATDKVAEKLSSTLSWVKNTVSHTVSQMASQVASPSTSLHTTSSSTTLSTPALSPSSPSQLSPDDLELLAKLEEQNRLLETDSKSLRSVNGSRRNSGSSLVSSSSASSNLSHLEEDSWILWGRIVNEWEDVRKKKEKQVKELVHKGIPHHFRAIVWQLLCSAQSMPIKDQYSELLKMTSPCEKLIRRDIARTYPEHNFFKEKDSLGQEVLFNVMKAYSLVDREVGYCQGSAFIVGLLLMQMPEEEAFCVFVKLMQDYRLRELFKPSMAELGLCMYQFECMIQEHLPELFVHFQSQSFHTSMYASSWFLTIFLTTFPLPIATRIFDIFMSEGLEIVFRVGLALLQMNQAELMQLDMEGMLQHFQKVIPHQFDGVPDKLIQAAYQVKYNSKKMKKLEKEYTTIKTKEMEEQVEIKRLRTENRLLKQRIETLEKHKCSSNYNEDFVLQLEKELVQARLSEAESQCALKEMQDKVLDIEKRNNSLPDENNIARLQEELIAVKLREAEAIMGLKELRQQVKDLEEHWQRHLARTTGRWKDPPKKNAMNELQDELMTIRLREAETQAEIREIKQRMMEMETQNQINSNHLRRAEQEVISLQEKVQYLSAQNKGLLTQLSEAKRKQAEIECKNKEEVMAVRLREADSIAAVAELRQHIAELEIQKEEGKLQGQLNKSDSNQYIGELKDQIAELNHELRCLKGQRGFSGQPPFDGIHIVNHLIGDDESFHSSDEDFIDNSLQETGVGFPLHGKSGSMSLDPAVADGSESETEDSVLETRESNQVVQKERPPRRRESYSTTV",
# NRP2_HUMAN
"MDMFPLTWVFLALYFSRHQVRGQPDPPCGGRLNSKDAGYITSPGYPQDYPSHQNCEWIVYAPEPNQKIVLNFNPHFEIEKHDCKYDFIEIRDGDSESADLLGKHCGNIAPPTIISSGSMLYIKFTSDYARQGAGFSLRYEIFKTGSEDCSKNFTSPNGTIESPGFPEKYPHNLDCTFTILAKPKMEIILQFLIFDLEHDPLQVGEGDCKYDWLDIWDGIPHVGPLIGKYCGTKTPSELRSSTGILSLTFHTDMAVAKDGFSARYYLVHQEPLENFQCNVPLGMESGRIANEQISASSTYSDGRWTPQQSRLHGDDNGWTPNLDSNKEYLQVDLRFLTMLTAIATQGAISRETQNGYYVKSYKLEVSTNGEDWMVYRHGKNHKVFQANNDATEVVLNKLHAPLLTRFVRIRPQTWHSGIALRLELFGCRVTDAPCSNMLGMLSGLIADSQISASSTQEYLWSPSAARLVSSRSGWFPRIPQAQPGEEWLQVDLGTPKTVKGVIIQGARGGDSITAVEARAFVRKFKVSYSLNGKDWEYIQDPRTQQPKLFEGNMHYDTPDIRRFDPIPAQYVRVYPERWSPAGIGMRLEVLGCDWTDSKPTVETLGPTVKSEETTTPYPTEEEATECGENCSFEDDKDLQLPSGFNCNFDFLEEPCGWMYDHAKWLRTTWASSSSPNDRTFPDDRNFLRLQSDSQREGQYARLISPPVHLPRSPVCMEFQYQATGGRGVALQVVREASQESKLLWVIREDQGGEWKHGRIILPSYDMEYQIVFEGVIGKGRSGEIAIDDIRISTDVPLENCMEPISAFAGENFKVDIPEIHEREGYEDEIDDEYEVDWSNSSSATSGSGAPSTDKEKSWLYTLDPILITIIAMSSLGVLLGATCAGLLLYCTCSYSGLSSRSCTTLENYNFELYDGLKHKVKMNHQKCCSEA",
# BRNP1_HUMAN
"MNWRFVELLYFLFIWGRISVQPSHQEPAGTDQHVSKEFDWLISDRGPFHHSRSYLSFVERHRQGFTTRYKIYREFARWKVRNTAIERRDLVRHPVPLMPEFQRSIRLLGRRPTTQQFIDTIIKKYGTHLLISATLGGEEALTMYMDKSRLDRKSGNATQSVEALHQLASSYFVDRDGTMRRLHEIQISTGAIKVTETRTGPLGCNSYDNLDSVSSVLLQSTESKLHLQGLQIIFPQYLQEKFVQSALSYIMCNGEGEYLCQNSQCRCQCAEEFPQCNCPITDIQIMEYTLANMAKSWAEAYKDLENSDEFKSFMKRLPSNHFLTIGSIHQHWGNDWDLQNRYKLLQSATEAQRQKIQRTARKLFGLSVRCRHNPNHQLPRERTIQQWLARVQSLLYCNENGFWGTFLESQRSCVCHGSTTLCQRPIPCVIGGNNSCAMCSLANISLCGSCNKGYKLYRGRCEPQNVDSERSEQFISFETDLDFQDLELKYLLQKMDSRLYVHTTFISNEIRLDTFFDPRWRKRMSLTLKSNKNRMDFIHMVIGMSMRICQMRNSSLDPMFFVYVNPFSGSHSEGWNMPFGEFGYPRWEKIRLQNSQCYNWTLLLGNRWKTFFETVHIYLRSRTRLPTLLRNETGQGPVDLSDPSKRQFYIKISDVQVFGYSLRFNADLLRSAVQQVNQSYTQGGQFYSSSSVMLLLLDIRDRINRLAPPVAPGKPQLDLFSCMLKHRLKLTNSEIIRVNHALDLYNTEILKQSDQMTAKLC",
# PLXC1_HUMAN
"MEVSRRKAPPRPPRPAAPLPLLAYLLALAAPGRGADEPVWRSEQAIGAIAASQEDGVFVASGSCLDQLDYSLEHSLSRLYRDQAGNCTEPVSLAPPARPRPGSSFSKLLLPYREGAAGLGGLLLTGWTFDRGACEVRPLGNLSRNSLRNGTEVVSCHPQGSTAGVVYRAGRNNRWYLAVAATYVLPEPETASRCNPAASDHDTAIALKDTEGRSLATQELGRLKLCEGAGSLHFVDAFLWNGSIYFPYYPYNYTSGAATGWPSMARIAQSTEVLFQGQASLDCGHGHPDGRRLLLSSSLVEALDVWAGVFSAAAGEGQERRSPTTTALCLFRMSEIQARAKRVSWDFKTAESHCKEGDQPERVQPIASSTLIHSDLTSVYGTVVMNRTVLFLGTGDGQLLKVILGENLTSNCPEVIYEIKEETPVFYKLVPDPVKNIYIYLTAGKEVRRIRVANCNKHKSCSECLTATDPHCGWCHSLQRCTFQGDCVHSENLENWLDISSGAKKCPKIQIIRSSKEKTTVTMVGSFSPRHSKCMVKNVDSSRELCQNKSQPNRTCTCSIPTRATYKDVSVVNVMFSFGSWNLSDRFNFTNCSSLKECPACVETGCAWCKSARRCIHPFTACDPSDYERNQEQCPVAVEKTSGGGRPKENKGNRTNQALQVFYIKSIEPQKVSTLGKSNVIVTGANFTRASNITMILKGTSTCDKDVIQVSHVLNDTHMKFSLPSSRKEMKDVCIQFDGGNCSSVGSLSYIALPHCSLIFPATTWISGGQNITMMGRNFDVIDNLIISHELKGNINVSEYCVATYCGFLAPSLKSSKVRTNVTVKLRVQDTYLDCGTLQYREDPRFTGYRVESEVDTELEVKIQKENDNFNISKKDIEITLFHGENGQLNCSFENITRNQDLTTILCKIKGIKTASTIANSSKKVRVKLGNLELYVEQESVPSTWYFLIVLPVLLVIVIFAAVGVTRHKSKELSRKQSQQLELLESELRKEIRDGFAELQMDKLDVVDSFGTVPFLDYKHFALRTFFPESGGFTHIFTEDMHNRDANDKNESLTALDALICNKSFLVTVIHTLEKQKNFSVKDRCLFASFLTIALQTKLVYLTSILEVLTRDLMEQCSNMQPKLMLRRTESVVEKLLTNWMSVCLSGFLRETVGEPFYLLVTTLNQKINKGPVDVITCKALYTLNEDWLLWQVPEFSTVALNVVFEKIPENESADVCRNISVNVLDCDTIGQAKEKIFQAFLSKNGSPYGLQLNEIGLELQMGTRQKELLDIDSSSVILEDGITKLNTIGHYEISNGSTIKVFKKIANFTSDVEYSDDHCHLILPDSEAFQDVQGKRHRGKHKFKVKEMYLTKLLSTKVAIHSVLEKLFRSIWSLPNSRAPFAIKYFFDFLDAQAENKKITDPDVVHIWKTNSLPLRFWVNILKNPQFVFDIKKTPHIDGCLSVIAQAFMDAFSLTEQQLGKEAPTNKLLYAKDIPTYKEEVKSYYKAIRDLPPLSSSEMEEFLTQESKKHENEFNEEVALTEIYKYIVKYFDEILNKLERERGLEEAQKQLLHVKVLFDEKKKCKWM",
# HNRPQ_HUMAN
"MATEHVNGNGTEEPMDTTSAVIHSENFQTLLDAGLPQKVAEKLDEIYVAGLVAHSDLDERAIEALKEFNEDGALAVLQQFKDSDLSHVQNKSAFLCGVMKTYRQREKQGTKVADSSKGPDEAKIKALLERTGYTLDVTTGQRKYGGPPPDSVYSGQQPSVGTEIFVGKIPRDLFEDELVPLFEKAGPIWDLRLMMDPLTGLNRGYAFVTFCTKEAAQEAVKLYNNHEIRSGKHIGVCISVANNRLFVGSIPKSKTKEQILEEFSKVTEGLTDVILYHQPDDKKKNRGFCFLEYEDHKTAAQARRRLMSGKVKVWGNVGTVEWADPIEDPDPEVMAKVKVLFVRNLANTVTEEILEKAFSQFGKLERVKKLKDYAFIHFDERDGAVKAMEEMNGKDLEGENIEIVFAKPPDQKRKERKAQRQAAKNQMYDDYYYYGPPHMPPPTRGRGRGGRGGYGYPPDYYGYEDYYDYYGYDYHNYRGGYEDPYYGYEDFQVGARGRGGRGARGAAPSRGRGAAPPRGRAGYSQRGGPGSARGVRGARGGAQQQRGRGVRGARGGRGGNVGGKRKADGYNQPDSKRRQTNNQNWGSQPIAQQPLQGGDHSGNYGYKSENQEFYQDTFGQQWK",
# RNBP6_HUMAN
"MAATASAGVPATVSEKQEFYQLLKNLINPSCMVRRQAEEIYENIPGLCKTTFLLDAVRNRRAGYEVRQMAAALLRRLLSSGFEEVYPNLPADVQRDVKIELILAVKLETHASMRKKLCDIFAVLARNLIDEDGTNHWPEGLKFLIDSIYSKNVVLWEVALHVFWHFPGIFGTQERHDLDIIKRLLDQCIQDQEHPAIRTLSARAAAAFVLANENNIALFKDFADLLPGILQAVNDSCYQDDDSVLESLVEIADTVPKYLGPYLEDTLQLSLKLCGDSRLSNLQRQLALEVIVTLSETATPMLKKHTNIIAQAVPHILAMMVDLQDDEDWVNADEMEEDDFDSNAVAAESALDRLACGLGGKVVLPMTKEHIMQMLQSPDWKYRHAGLMALSAIGEGCHQQMESILDETVNSVLLFLQDPHPRVRAAACTTLGQMATDFAPNFQKKFHETVIAALLRTMENQGNQRVQSHAASALIIFIEDCPKSLLVLYVDSMVKNLHSVLVIKLQELIRNGTKLALEQLVTTIASVADTIEEKFVPYYDIFMPSLKHIVELAVQKELKLLRGKTIECISHIGLAVGKEKFMQDASNVMQLLLKTQSDLNNMEDDDPQTSYMVSAWARMCKILGKDFQQYLPLVIEPLIKTASAKPDVALLDTQDVENMSDDDGWQFVNLGDQQSFGIKTSGLEAKATACQMLVYYAKELREGFVEYTEQVVKLMVPLLKFYFHDNVRVAAAESMPFLLECARIRGPEYLAQMWQFICDPLIKAIGTEPDTDVLSEIMNSFAKSIEVMGDGCLNDEHLEELGGILKAKLEGHFKNQELRQVKRQEENYDQQVEMSLQDEDECDVYILTKVSDILHSLFSTYKEKILPWFEQLLPLIVNLICSSRPWPDRQWGLCIFDDIIEHCSPTSFKYVEYFRWPMLLNMRDNNPEVRQAAAYGLGVMAQFGGDDYRSLCSEAVPLLVKVIKCANSKTKKNVIATENCISAIGKILKFKPNCVNVDEVLPHWLSWLPLHEDKEEAIQTLSFLCDLIESNHPVVIGPNNSNLPKIISIIAEGKINETINYEDPCAKRLANVVRQVQTSEDLWLECVSQLDDEQQEALQELLNFA",
# TDRD6_HUMAN
"MCSTPGMPAPGASLALRVSFVDVHPDVIPVQLWGLVGERRGEYLRLSREIQEAAATRGQWALGSASASPGELCLVQVGLLWHRCRVVSRQAQESRVFLLDEGRTITAGAGSLAPGRREFFNLPSEVLGCVLAGLVPAGCGAGSGEPPQHWPADAVDFLSNLQGKEVHGCVLDVLLLHRLVLLEVPDVFQQMRELGLARRVPDSLFRSLLERYLTAATASVGSGVPVLSRVPLKQKQPGLDYFYPQLQLGVTEAVVITQVCHPHRIHCQLRSVSQEIHRLSESMAQVYRGSTGTGDENSTSATWEEREESPDKPGSPCASCGLDGHWYRALLLETFRPQRCAQVLHVDYGRKELVSCSSLRYLLPEYFRMPVVTYPCALYGLWDGGRGWSRSQVGDLKTLILGKAVNAKIEFYCSFEHVYYVSLYGEDGINLNRVFGVQSCCLADRVLQSQATEEEEPETSQSQSPAEEVDEEISLPALRSIRLKMNAFYDAQVEFVKNPSEFWIRLRKHNVTFSKLMRRMCGFYSSASKLDGVVLKPEPDDLCCVKWKENGYYRAIVTKLDDKSVDVFLVDRGNSENVDWYDVRMLLPQFRQLPILAVKCTLADIWPLGKTWSQEAVSFFKKTVLHKELVIHILDKQDHQYVIEILDESRTGEENISKVIAQAGYAKYQEFETKENILVNAHSPGHVSNHFTTESNKIPFAKTGEGEQKAKRENKTTSVSKALSDTTVVTNGSTELVVQEKVKRASVYFPLMQNCLEIKPGSSSKGELEVGSTVEVRVSYVENPGYFWCQLTRNIQGLKTLMSDIQYYCKNTAAPHQRNTLACLAKRTVNRQWSRALISGIQSVEHVNVTFVDYGDREMVSVKNIYSISEEFLKVKAQAFRCSLYNLIQPVGQNPFVWDVKAIQAFNEFIDNAWQKNLELKCTIFALASINEELFNIVDLLTPFQSACHFLVEKRLARPVKLQKPLESSVQLHSYFYSTHDMKIGSEELVYITHIDDPWTFYCQLARNANILEQLSCSITQLSKVLLNLKTSPLNPGTLCLAKYTDGNWYRGIVIEKEPKKVFFVDFGNIYVVTSDDLLPIPSDAYDVLLLPMQAVRCSLSDIPDHIPEEVVVWFQETILDKSLKALVVAKDPDGTLIIELYGDNIQISASINKKLGLLSYKDRIRKKESEVLCSTTETLEEKNENMKLPCTEYLSKSVGYKLPNKEILEESYKPQINSSYKELKLLQSLTKTNLVTQYQDSVGNKNSQVFPLTTEKKEEISAETPLKTARVEATLSERKIGDSCDKDLPLKFCEFPQKTIMPGFKTTVYVSHINDLSDFYVQLIEDEAEISHLSERLNSVKTRPEYYVGPPLQRGDMICAVFPEDNLWYRAVIKEQQPNDLLSVQFIDYGNVSVVHTNKIGRLDLVNAILPGLCIHCSLQGFEVPDNKNSKKMMHYFSQRTSEAAIRCEFVKFQDRWEVILADEHGIIADDMISRYALSEKSQVELSTQVIKSASSKSVNKSDIDTSVFLNWYNPEKKMIRAYATVIDGPEYFWCQFADTEKLQCLEVEVQTAGEQVADRRNCIPCPYIGDPCIVRYREDGHYYRALITNICEDYLVSVRLVDFGNIEDCVDPKALWAIPSELLSVPMQAFPCCLSGFNISEGLCSQEGNDYFYEIITEDVLEITILEIRRDVCDIPLAIVDLKSKGKSINEKMEKYSKTGIKSALPYENIDSEIKQTLGSYNLDVGLKKLSNKAVQNKIYMEQQTDELAEITEKDVNIIGTKPSNFRDPKTDNICEGFENPCKDKIDTEELEGELECHLVDKAEFDDKYLITGFNTLLPHANETKEILELNSLEVPLSPDDESKEFLELESIELQNSLVVDEEKGELSPVPPNVPLSQECVTKGAMELFTLQLPLSCEAEKQPELELPTAQLPLDDKMDPLSLGVSQKAQESMCTEDMRKSSCVESFDDQRRMSLHLHGADCDPKTQNEMNICEEEFVEYKNRDAISALMPLFSEEESSDGSKHNNGLPDHISAQLQNTYTLKAFTVGSKCVVWSSLRNTWSKCEILETAEEGTRVLNLSNGMEEIVNPENVWNGIPKLDKSPPEKRGLEVMEI",
# DIAP1_HUMAN
"MEPPGGSLGPGRGTRDKKKGRSPDELPSAGGDGGKSKKFTLKRLMADELERFTSMRIKKEKEKPNSAHRNSSASYGDDPTAQSLQDVSDEQVLVLFEQMLLDMNLNEEKQQPLREKDIIIKREMVSQYLYTSKAGMSQKESSKSAMMYIQELRSGLRDMPLLSCLESLRVSLNNNPVSWVQTFGAEGLASLLDILKRLHDEKEETAGSYDSRNKHEIIRCLKAFMNNKFGIKTMLETEEGILLLVRAMDPAVPNMMIDAAKLLSALCILPQPEDMNERVLEAMTERAEMDEVERFQPLLDGLKSGTTIALKVGCLQLINALITPAEELDFRVHIRSELMRLGLHQVLQDLREIENEDMRVQLNVFDEQGEEDSYDLKGRLDDIRMEMDDFNEVFQILLNTVKDSKAEPHFLSILQHLLLVRNDYEARPQYYKLIEECISQIVLHKNGADPDFKCRHLQIEIEGLIDQMIDKTKVEKSEAKAAELEKKLDSELTARHELQVEMKKMESDFEQKLQDLQGEKDALHSEKQQIATEKQDLEAEVSQLTGEVAKLTKELEDAKKEMASLSAAAITVPPSVPSRAPVPPAPPLPGDSGTIIPPPPAPGDSTTPPPPPPPPPPPPPLPGGVCISSPPSLPGGTAISPPPPLSGDATIPPPPPLPEGVGIPSPSSLPGGTAIPPPPPLPGSARIPPPPPPLPGSAGIPPPPPPLPGEAGMPPPPPPLPGGPGIPPPPPFPGGPGIPPPPPGMGMPPPPPFGFGVPAAPVLPFGLTPKKLYKPEVQLRRPNWSKLVAEDLSQDCFWTKVKEDRFENNELFAKLTLTFSAQTKTSKAKKDQEGGEEKKSVQKKKVKELKVLDSKTAQNLSIFLGSFRMPYQEIKNVILEVNEAVLTESMIQNLIKQMPEPEQLKMLSELKDEYDDLAESEQFGVVMGTVPRLRPRLNAILFKLQFSEQVENIKPEIVSVTAACEELRKSESFSNLLEITLLVGNYMNAGSRNAGAFGFNISFLCKLRDTKSTDQKMTLLHFLAELCENDYPDVLKFPDELAHVEKASRVSAENLQKNLDQMKKQISDVERDVQNFPAATDEKDKFVEKMTSFVKDAQEQYNKLRMMHSNMETLYKELGEYFLFDPKKLSVEEFFMDLHNFRNMFLQAVKENQKRRETEEKMRRAKLAKEKAEKERLEKQQKREQLIDMNAEGDETGVMDSLLEALQSGAAFRRKRGPRQANRKAGCAVTSLLASELTKDDAMAAVPAKVSKNSETFPTILEEAKELVGRAS",
# AP180_HUMAN
"MSGQTLTDRIAAAQYSVTGSAVARAVCKATTHEVMGPKKKHLDYLIQATNETNVNIPQMADTLFERATNSSWVVVFKALVTTHHLMVHGNERFIQYLASRNTLFNLSNFLDKSGSHGYDMSTFIRRYSRYLNEKAFSYRQMAFDFARVKKGADGVMRTMAPEKLLKSMPILQGQIDALLEFDVHPNELTNGVINAAFMLLFKDLIKLFACYNDGVINLLEKFFEMKKGQCKDALEIYKRFLTRMTRVSEFLKVAEQVGIDKGDIPDLTQAPSSLMETLEQHLNTLEGKKPGNNEGSGAPSPLSKSSPATTVTSPNSTPAKTIDTSPPVDLFATASAAVPVSTSKPSSDLLDLQPDFSSGGAAAAAAPAPPPPAGGATAWGDLLGEDSLAALSSVPSEAQISDPFAPEPTPPTTTAEIATASASASTTTTVTAVTAEVDLFGDAFAASPGEAPAASEGAAAPATPTPVAAALDACSGNDPFAPSEGSAEAAPELDLFAMKPPETSVPVVTPTASTAPPVPATAPSPAPAVAAAAAATTAATAAATTTTTTSAATATTAPPALDIFGDLFESTPEVAAAPKPDAAPSIDLFSTDAFSSPPQGASPVPESSLTADLLSVDAFAAPSPATTASPAKVDSSGVIDLFGDAFGSSASEPQPASQAASSSSASADLLAGFGGSFMAPSPSPVTPAQNNLLQPNFEAAFGTTPSTSSSSSFDPSVFDGLGDLLMPTMAPAGQPAPVSMVPPSPAMAASKALGSDLDSSLASLVGNLGISGTTTKKGDLQWNAGEKKLTGGANWQPKVAPATWSAGVPPSAPLQGAVPPTSSVPPVAGAPSVGQPGAGFGMPPAGTGMPMMPQQPVMFAQPMMRPPFGAAAVPGTQLSPSPTPASQSPKKPPAKDPLADLNIKDFL",
# EXOC3_HUMAN
"MKETDREAVATAVQRVAGMLQRPDQLDKVEQYRRREARKKASVEARLKAAIQSQLDGVRTGLSQLHNALNDVKDIQQSLADVSKDWRQSINTIESLKDVKDAVVQHSQLAAAVENLKNIFSVPEIVRETQDLIEQGALLQAHRKLMDLECSRDGLMYEQYRMDSGNTRDMTLIHGYFGSTQGLSDELAKQLWMVLQRSLVTVRRDPTLLVSVVRIIEREEKIDRRILDRKKQTGFVPPGRPKNWKEKMFTILERTVTTRIEGTQADTRESDKMWLVRHLEIIRKYVLDDLIVAKNLMVQCFPPHYEIFKNLLNMYHQALSTRMQDLASEDLEANEIVSLLTWVLNTYTSTEMMRNVELAPEVDVGTLEPLLSPHVVSELLDTYMSTLTSNIIAWLRKALETDKKDWVKETEPEADQDGYYQTTLPAIVFQMFEQNLQVAAQISEDLKTKVLVLCLQQMNSFLSRYKDEAQLYKEEHLRNRQHPHCYVQYMIAIINNCQTFKESIVSLKRKYLKNEVEEGVSPSQPSMDGILDAIAKEGCSGLLEEVFLDLEQHLNELMTKKWLLGSNAVDIICVTVEDYFNDFAKIKKPYKKRMTAEAHRRVVVEYLRAVMQKRISFRSPEERKEGAEKMVREAEQLRFLFRKLASGFGEDVDGYCDTIVAVAEVIKLTDPSLLYLEVSTLVSKYPDIRDDHIGALLAVRGDASRDMKQTIMETLEQGPAQASPSYVPLFKDIVVPSLNVAKLLK",
# MOT2_HUMAN
"MPPMPSAPPVHPPPDGGWGWIVVGAAFISIGFSYAFPKAVTVFFKEIQQIFHTTYSEIAWISSIMLAVMYAGGPVSSVLVNKYGSRPVVIAGGLLCCLGMVLASFSSSVVQLYLTMGFITGLGLAFNLQPALTIIGKYFYRKRPMANGLAMAGSPVFLSSLAPFNQYLFNTFGWKGSFLILGSLLLNACVAGSLMRPLGPNQTTSKSKNKTGKTEDDSSPKKIKTKKSTWEKVNKYLDFSLFKHRGFLIYLSGNVIMFLGFFAPIIFLAPYAKDQGIDEYSAAFLLSVMAFVDMFARPSVGLIANSKYIRPRIQYFFSFAIMFNGVCHLLCPLAQDYTSLVLYAVFFGLGFGSVSSVLFETLMDLVGAPRFSSAVGLVTIVECGPVLLGPPLAGKLVDLTGEYKYMYMSCGAIVVAASVWLLIGNAINYRLLAKERKEENARQKTRESEPLSKSKHSEDVNVKVSNAQSVTSERETNI",
# JAK2_HUMAN
"MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHSLGKSEADYLTFPSGEYVAEEICIAASKACGITPVYHNMFALMSETERIWYPPNHVFHIDESTRHNVLYRIRFYFPRWYCSGSNRAYRHGISRGAEAPLLDDFVMSYLFAQWRHDFVHGWIKVPVTHETQEECLGMAVLDMMRIAKENDQTPLAIYNSISYKTFLPKCIRAKIQDYHILTRKRIRYRFRRFIQQFSQCKATARNLKLKYLINLETLQSAFYTEKFEVKEPGSGPSGEEIFATIIITGNGGIQWSRGKHKESETLTEQDLQLYCDFPNIIDVSIKQANQEGSNESRVVTIHKQDGKNLEIELSSLREALSFVSLIDGYYRLTADAHHYLCKEVAPPAVLENIQSNCHGPISMDFAISKLKKAGNQTGLYVLRCSPKDFNKYFLTFAVERENVIEYKHCLITKNENEEYNLSGTKKNFSSLKDLLNCYQMETVRSDNIIFQFTKCCPPKPKDKSNLLVFRTNGVSDVPTSPTLQRPTHMNQMVFHKIRNEDLIFNESLGQGTFTKIFKGVRREVGDYGQLHETEVLLKVLDKAHRNYSESFFEAASMMSKLSHKHLVLNYGVCVCGDENILVQEFVKFGSLDTYLKKNKNCINILWKLEVAKQLAWAMHFLEENTLIHGNVCAKNILLIREEDRKTGNPPFIKLSDPGISITVLPKDILQERIPWVPPECIENPKNLNLATDKWSFGTTLWEICSGGDKPLSALDSQRKLQFYEDRHQLPAPKWAELANLINNCMDYEPDFRPSFRAIIRDLNSLFTPDYELLTENDMLPNMRIGALGFSGAFEDRDPTQFEERHLKFLQQLGKGNFGSVEMCRYDPLQDNTGEVVAVKKLQHSTEEHLRDFEREIEILKSLQHDNIVKYKGVCYSAGRRNLKLIMEYLPYGSLRDYLQKHKERIDHIKLLQYTSQICKGMEYLGTKRYIHRDLATRNILVENENRVKIGDFGLTKVLPQDKEYYKVKEPGESPIFWYAPESLTESKFSVASDVWSFGVVLYELFTYIEKSKSPPAEFMRMIGNDKQGQMIVFHLIELLKNNGRLPRPDGCPDEIYMIMTECWNNNVNQRPSFRDLALRVDQIRDNMAG",
# SRPX2_HUMAN
"MASQLTQRGALFLLFFLTPAVTPTWYAGSGYYPDESYNEVYAEEVPQAPALDYRVPRWCYTLNIQDGEATCYSPKGGNYHSSLGTRCELSCDRGFRLIGRRSVQCLPSRRWSGTAYCRQMRCHALPFITSGTYTCTNGVLLDSRCDYSCSSGYHLEGDRSRICMEDGRWSGGEPVCVDIDPPKIRCPHSREKMAEPEKLTARVYWDPPLVKDSADGTITRVTLRGPEPGSHFPEGEHVIRYTAYDRAYNRASCKFIVKVQVRRCPTLKPPQHGYLTCTSAGDNYGATCEYHCDGGYDRQGTPSRVCQSSRQWSGSPPICAPMKINVNVNSAAGLLDQFYEKQRLLIISAPDPSNRYYKMQISMLQQSTCGLDLRHVTIIELVGQPPQEVGRIREQQLSANIIEELRQFQRLTRSYFNMVLIDKQGIDRDRYMEPVTPEEIFTFIDDYLLSNQELTQRREQRDICE",
# CTND1_HUMAN
"MDDSEVESTASILASVKEQEAQFEKLTRALEEERRHVSAQLERVRVSPQDANPLMANGTLTRRHQNGRFVGDADLERQKFSDLKLNGPQDHSHLLYSTIPRMQEPGQIVETYTEEDPEGAMSVVSVETSDDGTTRRTETTVKKVVKTVTTRTVQPVAMGPDGLPVDASSVSNNYIQTLGRDFRKNGNGGPGPYVGQAGTATLPRNFHYPPDGYSRHYEDGYPGGSDNYGSLSRVTRIEERYRPSMEGYRAPSRQDVYGPQPQVRVGGSSVDLHRFHPEPYGLEDDQRSMGYDDLDYGMMSDYGTARRTGTPSDPRRRLRSYEDMIGEEVPSDQYYWAPLAQHERGSLASLDSLRKGGPPPPNWRQPELPEVIAMLGFRLDAVKSNAAAYLQHLCYRNDKVKTDVRKLKGIPVLVGLLDHPKKEVHLGACGALKNISFGRDQDNKIAIKNCDGVPALVRLLRKARDMDLTEVITGTLWNLSSHDSIKMEIVDHALHALTDEVIIPHSGWEREPNEDCKPRHIEWESVLTNTAGCLRNVSSERSEARRKLRECDGLVDALIFIVQAEIGQKDSDSKLVENCVCLLRNLSYQVHREIPQAERYQEAAPNVANNTGPHAASCFGAKKGKDEWFSRGKKPIEDPANDTVDFPKRTSPARGYELLFQPEVVRIYISLLKESKTPAILEASAGAIQNLCAGRWTYGRYIRSALRQEKALSAIADLLTNEHERVVKAASGALRNLAVDARNKELIGKHAIPNLVKNLPGGQQNSSWNFSEDTVISILNTINEVIAENLEAAKKLRETQGIEKLVLINKSGNRSEKEVRAAALVLQTIWGYKELRKPLEKEGWKKSDFQVNLNNASRSQSSHSYDDSTLPLIDRNQKSDKKPDREEIQMSNMGSNTKSLDNNYSTPNERGDHNRTLDRSGDLGDMEPLKGTTPLMQDEGQESLEEELDVLVLDDEGGQVSYPSMQKI",
# HCN1_HUMAN
"MEGGGKPNSSSNSRDDGNSVFPAKASATGAGPAAAEKRLGTPPGGGGAGAKEHGNSVCFKVDGGGGGGGGGGGGEEPAGGFEDAEGPRRQYGFMQRQFTSMLQPGVNKFSLRMFGSQKAVEKEQERVKTAGFWIIHPYSDFRFYWDLIMLIMMVGNLVIIPVGITFFTEQTTTPWIIFNVASDTVFLLDLIMNFRTGTVNEDSSEIILDPKVIKMNYLKSWFVVDFISSIPVDYIFLIVEKGMDSEVYKTARALRIVRFTKILSLLRLLRLSRLIRYIHQWEEIFHMTYDLASAVVRIFNLIGMMLLLCHWDGCLQFLVPLLQDFPPDCWVSLNEMVNDSWGKQYSYALFKAMSHMLCIGYGAQAPVSMSDLWITMLSMIVGATCYAMFVGHATALIQSLDSSRRQYQEKYKQVEQYMSFHKLPADMRQKIHDYYEHRYQGKIFDEENILNELNDPLREEIVNFNCRKLVATMPLFANADPNFVTAMLSKLRFEVFQPGDYIIREGAVGKKMYFIQHGVAGVITKSSKEMKLTDGSYFGEICLLTKGRRTASVRADTYCRLYSLSVDNFNEVLEEYPMMRRAFETVAIDRLDRIGKKNSILLQKFQKDLNTGVFNNQENEILKQIVKHDREMVQAIAPINYPQMTTLNSTSSTTTPTSRMRTQSPPVYTATSLSHSNLHSPSPSTQTPQPSAILSPCSYTTAVCSPPVQSPLAARTFHYASPTASQLSLMQQQPQQQVQQSQPPQTQPQQPSPQPQTPGSSTPKNEVHKSTQALHNTNLTREVRPLSASQPSLPHEVSTLISRPHPTVGESLASIPQPVTAVPGTGLQAGGRSTVPQRVTLFRQMSSGAIPPNRGVPPAPPPPAAALPRESSSVLNTDPDAEKPRFASNL",
# PRAF2_HUMAN
"MSEVRLPPLRALDDFVLGSARLAAPDPCDPQRWCHRVINNLLYYQTNYLLCFGIGLALAGYVRPLHTLLSALVVAVALGVLVWAAETRAAVRRCRRSHPAACLAAVLAVGLLVLWVAGGACTFLFSIAGPVLLILVHASLRLRNLKNKIENKIESIGLKRTPMGLLLEALGQEQEAGS",
# IF2P_HUMAN
"MGKKQKNKSEDSTKDDIDLDALAAEIEGAGAAKEQEPQKSKGKKKKEKKKQDFDEDDILKELEELSLEAQGIKADRETVAVKPTENNEEEFTSKDKKKKGQKGKKQSFDDNDSEELEDKDSKSKKTAKPKVEMYSGSDDDDDFNKLPKKAKGKAQKSNKKWDGSEEDEDNSKKIKERSRINSSGESGDESDEFLQSRKGQKKNQKNKPGPNIESGNEDDDASFKIKTVAQKKAEKKERERKKRDEEKAKLRKLKEKEELETGKKDQSKQKESQRKFEEETVKSKVTVDTGVIPASEEKAETPTAAEDDNEGDKKKKDKKKKKGEKEEKEKEKKKGPSKATVKAMQEALAKLKEEEERQKREEEERIKRLEELEAKRKEEERLEQEKRERKKQKEKERKERLKKEGKLLTKSQREARARAEATLKLLQAQGVEVPSKDSLPKKRPIYEDKKRKKIPQQLESKEVSESMELCAAVEVMEQGVPEKEETPPPVEPEEEEDTEDAGLDDWEAMASDEETEKVEGNKVHIEVKENPEEEEEEEEEEEEDEESEEEEEEEGESEGSEGDEEDEKVSDEKDSGKTLDKKPSKEMSSDSEYDSDDDRTKEERAYDKAKRRIEKRRLEHSKNVNTEKLRAPIICVLGHVDTGKTKILDKLRHTHVQDGEAGGITQQIGATNVPLEAINEQTKMIKNFDRENVRIPGMLIIDTPGHESFSNLRNRGSSLCDIAILVVDIMHGLEPQTIESINLLKSKKCPFIVALNKIDRLYDWKKSPDSDVAATLKKQKKNTKDEFEERAKAIIVEFAQQGLNAALFYENKDPRTFVSLVPTSAHTGDGMGSLIYLLVELTQTMLSKRLAHCEELRAQVMEVKALPGMGTTIDVILINGRLKEGDTIIVPGVEGPIVTQIRGLLLPPPMKELRVKNQYEKHKEVEAAQGVKILGKDLEKTLAGLPLLVAYKEDEIPVLKDELIHELKQTLNAIKLEEKGVYVQASTLGSLEALLEFLKTSEVPYAGINIGPVHKKDVMKASVMLEHDPQYAVILAFDVRIERDAQEMADSLGVRIFSAEIIYHLFDAFTKYRQDYKKQKQEEFKHIAVFPCKIKILPQYIFNSRDPIVMGVTVEAGQVKQGTPMCVPSKNFVDIGIVTSIEINHKQVDVAKKGQEVCVKIEPIPGESPKMFGRHFEATDILVSKISRQSIDALKDWFRDEMQKSDWQLIVELKKVFEII",
# OPHN1_HUMAN
"MGHPPLEFSDCYLDSPDFRERLKCYEQELERTNKFIKDVIKDGNALISAMRNYSSAVQKFSQTLQSFQFDFIGDTLTDDEINIAESFKEFAELLNEVENERMMMVHNASDLLIKPLENFRKEQIGFTKERKKKFEKDGERFYSLLDRHLHLSSKKKESQLQEADLQVDKERHNFFESSLDYVYQIQEVQESKKFNIVEPVLAFLHSLFISNSLTVELTQDFLPYKQQLQLSLQNTRNHFSSTREEMEELKKRMKEAPQTCKLPGQPTIEGYLYTQEKWALGISWVKYYCQYEKETKTLTMTPMEQKPGAKQGPLDLTLKYCVRRKTESIDKRFCFDIETNERPGTITLQALSEANRRLWMEAMDGKEPIYHSPITKQQEMELNEVGFKFVRKCINIIETKGIKTEGLYRTVGSNIQVQKLLNAFFDPKCPGDVDFHNSDWDIKTITSSLKFYLRNLSEPVMTYRLHKELVSAAKSDNLDYRLGAIHSLVYKLPEKNREMLELLIRHLVNVCEHSKENLMTPSNMGVIFGPTLMRAQEDTVAAMMNIKFQNIVVEILIEHFGKIYLGPPEESAAPPVPPPRVTARRHKPITISKRLLRERTVFYTSSLDESEDEIQHQTPNGTITSSIEPPKPPQHPKLPIQRSGETDPGRKSPSRPILDGKLEPCPEVDVGKLVSRLQDGGTKITPKATNGPMPGSGPTKTPSFHIKRPAPRPLAHHKEGDADSFSKVRPPGEKPTIIRPPVRPPDPPCRAATPQKPEPKPDIVAGNAGEITSSVVASRTRFFETASRKTGSSQGRLPGDES",
# RAMP1_HUMAN
"MARALCRLPRRGLWLLLAHHLFMTTACQEANYGALLRELCLTQFQVDMEAVGETLWCDWGRTIRSYRELADCTWHMAEKLGCFWPNAEVDRFFLAVHGRYFRSCPISGRAVRDPPGSILYPFIVVPITVTLLVTALVVWQSKRTEGIV",
# NOL3_HUMAN
"MGNAQERPSETIDRERKRLVETLQADSGLLLDALLARGVLTGPEYEALDALPDAERRVRRLLLLVQGKGEAACQELLRCAQRTAGAPDPAWDWQHVGPGYRDRSYDPPCPGHWTPEAPGSGTTCPGLPRASDPDEAGGPEGSEAVQSGTPEEPEPELEAEASKEAEPEPEPEPELEPEAEAEPEPELEPEPDPEPEPDFEERDESEDS",
# DTNB_HUMAN
"MIEESGNKRKTMAEKRQLFIEMRAQNFDVIRLSTYRTACKLRFVQKRCNLHLVDIWNMIEAFRDNGLNTLDHTTEISVSRLETVISSIYYQLNKRLPSTHQISVEQSISLLLNFMIAAYDSEGRGKLTVFSVKAMLATMCGGKMLDKLRYVFSQMSDSNGLMIFSKFDQFLKEVLKLPTAVFEGPSFGYTEHSVRTCFPQQRKIMLNMFLDTMMADPPPQCLVWLPLMHRLAHVENVFHPVECSYCRCESMMGFRYRCQQCHNYQLCQNCFWRGHAGGPHSNQHQMKEHSSWKSPAKKLSHAISKSLGCVPTREPPHPVFPEQPEKPLDLAHIVPPRPLTNMNDTMVSHMSSGVPTPTKRLQYSQDIPSHLADEHALIASYVARLQHCARVLDSPSRLDEEHRLIARYAARLAAEAGNVTRPPTDLSFNFDANKQQRQLIAELENKNREILQEIQRLRLEHEQASQPTPEKAQQNPTLLAELRLLRQRKDELEQRMSALQESRRELMVQLEELMKLLKEEEQKQAAQATGSPHTSPTHGGGRPMPMPVRSTSAGSTPTHCPQDSLSGVGGDVQEAFAQGTRRNLRNDLLVAADSITNTMSSLVKELHSAEEGAEEEEEKMQNGKDRG",
# KI21B_HUMAN
"MAGQGDCCVKVAVRIRPQLSKEKIEGCHICTSVTPGEPQVLLGKDKAFTYDFVFDLDTWQEQIYSTCVSKLIEGCFEGYNATVLAYGQTGAGKTYTMGTGFDMATSEEEQGIIPRAIAHLFGGIAERKRRAQEQGVAGPEFKVSAQFLELYNEEILDLFDSTRDPDTRHRRSNIKIHEDANGGIYTTGVTSRLIHSQEELIQCLKQGALSRTTASTQMNVQSSRSHAIFTIHLCQMRMCTQPDLVNEAVTGLPDGTPPSSEYETLTAKFHFVDLAGSERLKRTGATGERAKEGISINCGLLALGNVISALGDQSKKVVHVPYRDSKLTRLLQDSLGGNSQTIMIACVSPSDRDFMETLNTLKYANRARNIKNKVVVNQDKTSQQISALRAEIARLQMELMEYKAGKRVIGEDGAEGYSDLFRENAMLQKENGALRLRVKAMQEAIDAINNRVTQLMSQEANLLLAKAGDGNEAIGALIQNYIREIEELRTKLLESEAMNESLRRSLSRASARSPYSLGASPAAPAFGGSPASSMEDASEVIRRAKQDLERLKKKEVRQRRKSPEKEAFKKRAKLQQENSEETDENEAEEEEEERDESGCEEEEGREDEDEDSGSEESLVDSDSDPEEKEVNFQADLADLTCEIEIKQKLIDELENSQRRLQTLKHQYEEKLILLQNKIRDTQLERDRVLQNLSTMECYTEEKANKIKADYEKRLREMNRDLQKLQAAQKEHARLLKNQSRYERELKKLQAEVAEMKKAKVALMKQMREEQQRRRLVETKRNREIAQLKKEQRRQEFQIRALESQKRQQEMVLRRKTQEVSALRRLAKPMSERVAGRAGLKPPMLDSGAEVSASTTSSEAESGARSVSSIVRQWNRKINHFLGDHPAPTVNGTRPARKKFQKKGASQSFSKAARLKWQSLERRIIDIVMQRMTIVNLEADMERLIKKREELFLLQEALRRKRERLQAESPEEEKGLQELAEEIEVLAANIDYINDGITDCQATIVQLEETKEELDSTDTSVVISSCSLAEARLLLDNFLKASIDKGLQVAQKEAQIRLLEGRLRQTDMAGSSQNHLLLDALREKAEAHPELQALIYNVQQENGYASTDEEISEFSEGSFSQSFTMKGSTSHDDFKFKSEPKLSAQMKAVSAECLGPPLDISTKNITKSLASLVEIKEDGVGFSVRDPYYRDRVSRTVSLPTRGSTFPRQSRATETSPLTRRKSYDRGQPIRSTDVGFTPPSSPPTRPRNDRNVFSRLTSNQSQGSALDKSDDSDSSLSEVLRGIISPVGGAKGARTAPLQCVSMAEGHTKPILCLDATDELLFTGSKDRSCKMWNLVTGQEIAALKGHPNNVVSIKYCSHSGLVFSVSTSYIKVWDIRDSAKCIRTLTSSGQVISGDACAATSTRAITSAQGEHQINQIALSPSGTMLYAASGNAVRIWELSRFQPVGKLTGHIGPVMCLTVTQTASQHDLVVTGSKDHYVKMFELGECVTGTIGPTHNFEPPHYDGIECLAIQGDILFSGSRDNGIKKWDLDQQELIQQIPNAHKDWVCALAFIPGRPMLLSACRAGVIKVWNVDNFTPIGEIKGHDSPINAICTNAKHIFTASSDCRVKLWNYVPGLTPCLPRRVLAIKGRATTLP",
# SRGP2_HUMAN
"MTSPAKFKKDKEIIAEYDTQVKEIRAQLTEQMKCLDQQCELRVQLLQDLQDFFRKKAEIEMDYSRNLEKLAERFLAKTRSTKDQQFKKDQNVLSPVNCWNLLLNQVKRESRDHTTLSDIYLNNIIPRFVQVSEDSGRLFKKSKEVGQQLQDDLMKVLNELYSVMKTYHMYNADSISAQSKLKEAEKQEEKQIGKSVKQEDRQTPRSPDSTANVRIEEKHVRRSSVKKIEKMKEKRQAKYTENKLKAIKARNEYLLALEATNASVFKYYIHDLSDLIDQCCDLGYHASLNRALRTFLSAELNLEQSKHEGLDAIENAVENLDATSDKQRLMEMYNNVFCPPMKFEFQPHMGDMASQLCAQQPVQSELVQRCQQLQSRLSTLKIENEEVKKTMEATLQTIQDIVTVEDFDVSDCFQYSNSMESVKSTVSETFMSKPSIAKRRANQQETEQFYFTKMKEYLEGRNLITKLQAKHDLLQKTLGESQRTDCSLARRSSTVRKQDSSQAIPLVVESCIRFISRHGLQHEGIFRVSGSQVEVNDIKNAFERGEDPLAGDQNDHDMDSIAGVLKLYFRGLEHPLFPKDIFHDLMACVTMDNLQERALHIRKVLLVLPKTTLIIMRYLFAFLNHLSQFSEENMMDPYNLAICFGPSLMSVPEGHDQVSCQAHVNELIKTIIIQHENIFPSPRELEGPVYSRGGSMEDYCDSPHGETTSVEDSTQDVTAEHHTSDDECEPIEAIAKFDYVGRTARELSFKKGASLLLYQRASDDWWEGRHNGIDGLIPHQYIVVQDTEDGVVERSSPKSEIEVISEPPEEKVTARAGASCPSGGHVADIYLANINKQRKRPESGSIRKTFRSDSHGLSSSLTDSSSPGVGASCRPSSQPIMSQSLPKEGPDKCSISGHGSLNSISRHSSLKNRLDSPQIRKTATAGRSKSFNNHRPMDPEVIAQDIEATMNSALNELRELERQSSVKHTPDVVLDTLEPLKTSPVVAPTSEPSSPLHTQLLKDPEPAFQRSASTAGDIACAFRPVKSVKMAAPVKPPATRPKPTVFPKTNATSPGVNSSTSPQSTDKSCTV",
# CAPON_HUMAN
"MPSKTKYNLVDDGHDLRIPLHNEDAFQHGICFEAKYVGSLDVPRPNSRVEIVAAMRRIRYEFKAKNIKKKKVSIMVSVDGVKVILKKKKKLLLLQKKEWTWDESKMLVMQDPIYRIFYVSHDSQDLKIFSYIARDGASNIFRCNVFKSKKKSQAMRIVRTVGQAFEVCHKLSLQHTQQNADGQEDGESERNSNSSGDPGRQLTGAERASTATAEETDIDAVEVPLPGNDVLEFSRGVTDLDAVGKEGGSHTGSKVSHPQEPMLTASPRMLLPSSSSKPPGLGTETPLSTHHQMQLLQQLLQQQQQQTQVAVAQVHLLKDQLAAEAAARLEAQARVHQLLLQNKDMLQHISLLVKQVQELELKLSGQNAMGSQDSLLEITFRSGALPVLCDPTTPKPEDLHSPPLGAGLADFAHPAGSPLGRRDCLVKLECFRFLPPEDTPPPAQGEALLGGLELIKFRESGIASEYESNTDESEERDSWSQEELPRLLNVLQRQELGDGLDDEIAV",
# AUXI_HUMAN
"MKDSENKGASSPDMEPSYGGGLFDMVKGGAGRLFSNLKDNLKDTLKDTSSRVIQSVTSYTKGDLDFTYVTSRIIVMSFPLDNVDIGFRNQVDDIRSFLDSRHLDHYTVYNLSPKSYRTAKFHSRVSECSWPIRQAPSLHNLFAVCRNMYNWLLQNPKNVCVVHCLDGRAASSILVGAMFIFCNLYSTPGPAIRLLYAKRPGIGLSPSHRRYLGYMCDLLADKPYRPHFKPLTIKSITVSPIPFFNKQRNGCRPYCDVLIGETKIYSTCTDFERMKEYRVQDGKIFIPLNITVQGDVVVSMYHLRSTIGSRLQAKVTNTQIFQLQFHTGFIPLDTTVLKFTKPELDACDVPEKYPQLFQVTLDVELQPHDKVIDLTPPWEHYCTKDVNPSILFSSHQEHQDTLALGGQAPIDIPPDNPRHYGQSGFFASLCWQDQKSEKSFCEEDHAALVNQESEQSDDELLTLSSPHGNANGDKPHGVKKPSKKQQEPAAPPPPEDVDLLGLEGSAMSNSFSPPAAPPTNSELLSDLFGGGGAAGPTQAGQSGVEDVFHPSGPASTQSTPRRSATSTSASPTLRVGEGATFDPFGAPSKPSGQDLLGSFLNTSSASSDPFLQPTRSPSPTVHASSTPAVNIQPDVSGGWDWHAKPGGFGMGSKSAATSPTGSSHGTPTHQSKPQTLDPFADLGTLGSSSFASKPTTPTGLGGGFPPLSSPQKASPQPMGGGWQQGGAYNWQQPQPKPQPSMPHSSPQNRPNYNVSFSAMPGGQNERGKGSSNLEGKQKAADFEDLLSGQGFNAHKDKKGPRTIAEMRKEEMAKEMDPEKLKILEWIEGKERNIRALLSTMHTVLWAGETKWKPVGMADLVTPEQVKKVYRKAVLVVHPDKATGQPYEQYAKMIFMELNDAWSEFENQGQKPLY",
# ADA23_HUMAN
"MKPPGSSSRQPPLAGCSLAGASCGPQRGPAGSVPASAPARTPPCRLLLVLLLLPPLAASSRPRAWGAAAPSAPHWNETAEKNLGVLADEDNTLQQNSSSNISYSNAMQKEITLPSRLIYYINQDSESPYHVLDTKARHQQKHNKAVHLAQASFQIEAFGSKFILDLILNNGLLSSDYVEIHYENGKPQYSKGGEHCYYHGSIRGVKDSKVALSTCNGLHGMFEDDTFVYMIEPLELVHDEKSTGRPHIIQKTLAGQYSKQMKNLTMERGDQWPFLSELQWLKRRKRAVNPSRGIFEEMKYLELMIVNDHKTYKKHRSSHAHTNNFAKSVVNLVDSIYKEQLNTRVVLVAVETWTEKDQIDITTNPVQMLHEFSKYRQRIKQHADAVHLISRVTFHYKRSSLSYFGGVCSRTRGVGVNEYGLPMAVAQVLSQSLAQNLGIQWEPSSRKPKCDCTESWGGCIMEETGVSHSRKFSKCSILEYRDFLQRGGGACLFNRPTKLFEPTECGNGYVEAGEECDCGFHVECYGLCCKKCSLSNGAHCSDGPCCNNTSCLFQPRGYECRDAVNECDITEYCTGDSGQCPPNLHKQDGYACNQNQGRCYNGECKTRDNQCQYIWGTKAAGSDKFCYEKLNTEGTEKGNCGKDGDRWIQCSKHDVFCGFLLCTNLTRAPRIGQLQGEIIPTSFYHQGRVIDCSGAHVVLDDDTDVGYVEDGTPCGPSMMCLDRKCLQIQALNMSSCPLDSKGKVCSGHGVCSNEATCICDFTWAGTDCSIRDPVRNLHPPKDEGPKGPSATNLIIGSIAGAILVAAIVLGGTGWGFKNVKKRRFDPTQQGPI",
# WDR1_HUMAN
"MPYEIKKVFASLPQVERGVSKIIGGDPKGNNFLYTNGKCVILRNIDNPALADIYTEHAHQVVVAKYAPSGFYIASGDVSGKLRIWDTTQKEHLLKYEYQPFAGKIKDIAWTEDSKRIAVVGEGREKFGAVFLWDSGSSVGEITGHNKVINSVDIKQSRPYRLATGSDDNCAAFFEGPPFKFKFTIGDHSRFVNCVRFSPDGNRFATASADGQIYIYDGKTGEKVCALGGSKAHDGGIYAISWSPDSTHLLSASGDKTSKIWDVSVNSVVSTFPMGSTVLDQQLGCLWQKDHLLSVSLSGYINYLDRNNPSKPLHVIKGHSKSIQCLTVHKNGGKSYIYSGSHDGHINYWDSETGENDSFAGKGHTNQVSRMTVDESGQLISCSMDDTVRYTSLMLRDYSGQGVVKLDVQPKCVAVGPGGYAVVVCIGQIVLLKDQRKCFSIDNPGYEPEVVAVHPGGDTVAIGGVDGNVRLYSILGTTLKDEGKLLEAKGPVTDVAYSHDGAFLAVCDASKVVTVFSVADGYSENNVFYGHHAKIVCLAWSPDNEHFASGGMDMMVYVWTLSDPETRVKIQDAHRLHHVSSLAWLDEHTLVTTSHDASVKEWTITY",
# LRP4_HUMAN
"MRRQWGALLLGALLCAHGLASSPECACGRSHFTCAVSALGECTCIPAQWQCDGDNDCGDHSDEDGCILPTCSPLDFHCDNGKCIRRSWVCDGDNDCEDDSDEQDCPPRECEEDEFPCQNGYCIRSLWHCDGDNDCGDNSDEQCDMRKCSDKEFRCSDGSCIAEHWYCDGDTDCKDGSDEENCPSAVPAPPCNLEEFQCAYGRCILDIYHCDGDDDCGDWSDESDCSSHQPCRSGEFMCDSGLCINAGWRCDGDADCDDQSDERNCTTSMCTAEQFRCHSGRCVRLSWRCDGEDDCADNSDEENCENTGSPQCALDQFLCWNGRCIGQRKLCNGVNDCGDNSDESPQQNCRPRTGEENCNVNNGGCAQKCQMVRGAVQCTCHTGYRLTEDGHTCQDVNECAEEGYCSQGCTNSEGAFQCWCETGYELRPDRRSCKALGPEPVLLFANRIDIRQVLPHRSEYTLLLNNLENAIALDFHHRRELVFWSDVTLDRILRANLNGSNVEEVVSTGLESPGGLAVDWVHDKLYWTDSGTSRIEVANLDGAHRKVLLWQNLEKPRAIALHPMEGTIYWTDWGNTPRIEASSMDGSGRRIIADTHLFWPNGLTIDYAGRRMYWVDAKHHVIERANLDGSHRKAVISQGLPHPFAITVFEDSLYWTDWHTKSINSANKFTGKNQEIIRNKLHFPMDIHTLHPQRQPAGKNRCGDNNGGCTHLCLPSGQNYTCACPTGFRKISSHACAQSLDKFLLFARRMDIRRISFDTEDLSDDVIPLADVRSAVALDWDSRDDHVYWTDVSTDTISRAKWDGTGQEVVVDTSLESPAGLAIDWVTNKLYWTDAGTDRIEVANTDGSMRTVLIWENLDRPRDIVVEPMGGYMYWTDWGASPKIERAGMDASGRQVIISSNLTWPNGLAIDYGSQRLYWADAGMKTIEFAGLDGSKRKVLIGSQLPHPFGLTLYGERIYWTDWQTKSIQSADRLTGLDRETLQENLENLMDIHVFHRRRPPVSTPCAMENGGCSHLCLRSPNPSGFSCTCPTGINLLSDGKTCSPGMNSFLIFARRIDIRMVSLDIPYFADVVVPINITMKNTIAIGVDPQEGKVYWSDSTLHRISRANLDGSQHEDIITTGLQTTDGLAVDAIGRKVYWTDTGTNRIEVGNLDGSMRKVLVWQNLDSPRAIVLYHEMGFMYWTDWGENAKLERSGMDGSDRAVLINNNLGWPNGLTVDKASSQLLWADAHTERIEAADLNGANRHTLVSPVQHPYGLTLLDSYIYWTDWQTRSIHRADKGTGSNVILVRSNLPGLMDMQAVDRAQPLGFNKCGSRNGGCSHLCLPRPSGFSCACPTGIQLKGDGKTCDPSPETYLLFSSRGSIRRISLDTSDHTDVHVPVPELNNVISLDYDSVDGKVYYTDVFLDVIRRADLNGSNMETVIGRGLKTTDGLAVDWVARNLYWTDTGRNTIEASRLDGSCRKVLINNSLDEPRAIAVFPRKGYLFWTDWGHIAKIERANLDGSERKVLINTDLGWPNGLTLDYDTRRIYWVDAHLDRIESADLNGKLRQVLVSHVSHPFALTQQDRWIYWTDWQTKSIQRVDKYSGRNKETVLANVEGLMDIIVVSPQRQTGTNACGVNNGGCTHLCFARASDFVCACPDEPDSRPCSLVPGLVPPAPRATGMSEKSPVLPNTPPTTLYSSTTRTRTSLEEVEGRCSERDARLGLCARSNDAVPAAPGEGLHISYAIGGLLSILLILVVIAALMLYRHKKSKFTDPGMGNLTYSNPSYRTSTQEVKIEAIPKPAMYNQLCYKKEGGPDHNYTKEKIKIVEGICLLSGDDAEWDDLKQLRSSRGGLLRDHVCMKTDTVSIQASSGSLDDTETEQLLQEEQSECSSVHTAATPERRGSLPDTGWKHERKLSSESQV",
# ROCK2_HUMAN
"MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESLLDGLNSLVLDLDFPALRKNKNIDNFLNRYEKIVKKIRGLQMKAEDYDVVKVIGRGAFGEVQLVRHKASQKVYAMKLLSKFEMIKRSDSAFFWEERDIMAFANSPWVVQLFYAFQDDRYLYMVMEYMPGGDLVNLMSNYDVPEKWAKFYTAEVVLALDAIHSMGLIHRDVKPDNMLLDKHGHLKLADFGTCMKMDETGMVHCDTAVGTPDYISPEVLKSQGGDGFYGRECDWWSVGVFLYEMLVGDTPFYADSLVGTYSKIMDHKNSLCFPEDAEISKHAKNLICAFLTDREVRLGRNGVEEIRQHPFFKNDQWHWDNIRETAAPVVPELSSDIDSSNFDDIEDDKGDVETFPIPKAFVGNQLPFIGFTYYRENLLLSDSPSCRETDSIQSRKNEESQEIQKKLYTLEEHLSNEMQAKEELEQKCKSVNTRLEKTAKELEEEITLRKSVESALRQLEREKALLQHKNAEYQRKADHEADKKRNLENDVNSLKDQLEDLKKRNQNSQISTEKVNQLQRQLDETNALLRTESDTAARLRKTQAESSKQIQQLESNNRDLQDKNCLLETAKLKLEKEFINLQSALESERRDRTHGSEIINDLQGRICGLEEDLKNGKILLAKVELEKRQLQERFTDLEKEKSNMEIDMTYQLKVIQQSLEQEEAEHKATKARLADKNKIYESIEEAKSEAMKEMEKKLLEERTLKQKVENLLLEAEKRCSLLDCDLKQSQQKINELLKQKDVLNEDVRNLTLKIEQETQKRCLTQNDLKMQTQQVNTLKMSEKQLKQENNHLMEMKMNLEKQNAELRKERQDADGQMKELQDQLEAEQYFSTLYKTQVRELKEECEEKTKLGKELQQKKQELQDERDSLAAQLEITLTKADSEQLARSIAEEQYSDLEKEKIMKELEIKEMMARHKQELTEKDATIASLEETNRTLTSDVANLANEKEELNNKLKDVQEQLSRLKDEEISAAAIKAQFEKQLLTERTLKTQAVNKLAEIMNRKEPVKRGNDTDVRRKEKENRKLHMELKSEREKLTQQMIKYQKELNEMQAQIAEESQIRIELQMTLDSKDSDIEQLRSQLQALHIGLDSSSIGSGPGDAEADDGFPESRLEGWLSLPVRNNTKKFGWVKKYVIVSSKKILFYDSEQDKEQSNPYMVLDIDKLFHVRPVTQTDVYRADAKEIPRIFQILYANEGESKKEQEFPVEPVGEKSNYICHKGHEFIPTLYHFPTNCEACMKPLWHMFKPPPALECRRCHIKCHKDHMDKKEEIIAPCKVYYDISTAKNLLLLANSTEEQQKWVSRLVKKIPKKPPAPDPFARSSPRTSMKIQQNQSIRRPSRQLAPNKPS",
# CLAP2_HUMAN
"MAMGDDKSFDDEESVDGNRPSSAASAFKVPAPKTSGNPANSARKPGSAGGPKVGGASKEGGAGAVDEDDFIKAFTDVPSIQIYSSRELEETLNKIREILSDDKHDWDQRANALKKIRSLLVAGAAQYDCFFQHLRLLDGALKLSAKDLRSQVVREACITVAHLSTVLGNKFDHGAEAIVPTLFNLVPNSAKVMATSGCAAIRFIIRHTHVPRLIPLITSNCTSKSVPVRRRSFEFLDLLLQEWQTHSLERHAAVLVETIKKGIHDADAEARVEARKTYMGLRNHFPGEAETLYNSLEPSYQKSLQTYLKSSGSVASLPQSDRSSSSSQESLNRPFSSKWSTANPSTVAGRVSAGSSKASSLPGSLQRSRSDIDVNAAAGAKAHHAAGQSVRSGRLGAGALNAGSYASLEDTSDKLDGTASEDGRVRAKLSAPLAGMGNAKADSRGRSRTKMVSQSQPGSRSGSPGRVLTTTALSTVSSGVQRVLVNSASAQKRSKIPRSQGCSREASPSRLSVARSSRIPRPSVSQGCSREASRESSRDTSPVRSFQPLASRHHSRSTGALYAPEVYGASGPGYGISQSSRLSSSVSAMRVLNTGSDVEEAVADALKKPARRRYESYGMHSDDDANSDASSACSERSYSSRNGSIPTYMRQTEDVAEVLNRCASSNWSERKEGLLGLQNLLKNQRTLSRVELKRLCEIFTRMFADPHGKRVFSMFLETLVDFIQVHKDDLQDWLFVLLTQLLKKMGADLLGSVQAKVQKALDVTRESFPNDLQFNILMRFTVDQTQTPSLKVKVAILKYIETLAKQMDPGDFINSSETRLAVSRVITWTTEPKSSDVRKAAQSVLISLFELNTPEFTMLLGALPKTFQDGATKLLHNHLRNTGNGTQSSMGSPLTRPTPRSPANWSSPLTSPTNTSQNTLSPSAFDYDTENMNSEDIYSSLRGVTEAIQNFSFRSQEDMNEPLKRDSKKDDGDSMCGGPGMSDPRAGGDATDSSQTALDNKASLLHSMPTHSSPRSRDYNPYNYSDSISPFNKSALKEAMFDDDADQFPDDLSLDHSDLVAELLKELSNHNERVEERKIALYELMKLTQEESFSVWDEHFKTILLLLLETLGDKEPTIRALALKVLREILRHQPARFKNYAELTVMKTLEAHKDPHKEVVRSAEEAASVLATSISPEQCIKVLCPIIQTADYPINLAAIKMQTKVIERVSKETLNLLLPEIMPGLIQGYDNSESSVRKACVFCLVAVHAVIGDELKPHLSQLTGSKMKLLNLYIKRAQTGSGGADPTTDVSGQS",
# LIPA3_HUMAN
"MMCEVMPTISEDGRRGSALGPDEAGGELERLMVTMLTERERLLETLREAQDGLATAQLRLRELGHEKDSLQRQLSIALPQEFAALTKELNLCREQLLEREEEIAELKAERNNTRLLLEHLECLVSRHERSLRMTVVKRQAQSPGGVSSEVEVLKALKSLFEHHKALDEKVRERLRMALERVAVLEEELELSNQETLNLREQLSRRRSGLEEPGKDGDGQTLANGLGPGGDSNRRTAELEEALERQRAEVCQLRERLAVLCRQMSQLEEELGTAHRELGKAEEANSKLQRDLKEALAQREDMEERITTLEKRYLSAQREATSLHDANDKLENELASKESLYRQSEEKSRQLAEWLDDAKQKLQQTLQKAETLPEIEAQLAQRVAALNKAEERHGNFEERLRQLEAQLEEKNQELQRARQREKMNDDHNKRLSETVDKLLSESNERLQLHLKERMGALEEKNSLSEEIANMKKLQDELLLNKEQLLAEMERMQMEIDQLRGRPPSSYSRSLPGSALELRYSQAPTLPSGAHLDPYVAGSGRAGKRGRWSGVKEEPSKDWERSAPAGSIPPPFPGELDGSDEEEAEGMFGAELLSPSGQADVQTLAIMLQEQLEAINKEIKLIQEEKETTEQRAEELESRVSSSGLDSLGRYRSSCSLPPSLTTSTLASPSPPSSGHSTPRLAPPSPAREGTDKANHVPKEEAGAPRGEGPAIPGDTPPPTPRSARLERMTQALALQAGSLEDGGPPRGSEGTPDSLHKAPKKKSIKSSIGRLFGKKEKGRMGPPGRDSSSLAGTPSDETLATDPLGLAKLTGPGDKDRRNKRKHELLEEACRQGLPFAAWDGPTVVSWLELWVGMPAWYVAACRANVKSGAIMANLSDTEIQREIGISNPLHRLKLRLAIQEMVSLTSPSAPASSRTSTGNVWMTHEEMESLTATTKPETKEISWEQILAYGDMNHEWVGNDWLPSLGLPQYRSYFMESLVDARMLDHLNKKELRGQLKMVDSFHRVSLHYGIMCLKRLNYDRKDLERRREESQTQIRDVMVWSNERVMGWVSGLGLKEFATNLTESGVHGALLALDETFDYSDLALLLQIPTQNAQARQLLEKEFSNLISLGTDRRLDEDSAKSFSRSPSWRKMFREKDLRGVTPDSAEMLPPNFRSAAAGALGSPGLPLRKLQPEGQTSGSSRADGVSVRTYSC",
# HIP1R_HUMAN
"MNSIKNVPARVLSRRPGHSLEAEREQFDKTQAISISKAINTQEAPVKEKHARRIILGTHHEKGAFTFWSYAIGLPLPSSSILSWKFCHVLHKVLRDGHPNVLHDCQRYRSNIREIGDLWGHLHDRYGQLVNVYTKLLLTKISFHLKHPQFPAGLEVTDEVLEKAAGTDVNNIFQLTVEMFDYMDCELKLSESVFRQLNTAIAVSQMSSGQCRLAPLIQVIQDCSHLYHYTVKLLFKLHSCLPADTLQGHRDRFHEQFHSLRNFFRRASDMLYFKRLIQIPRLPEGPPNFLRASALAEHIKPVVVIPEEAPEDEEPENLIEISTGPPAGEPVVVADLFDQTFGPPNGSVKDDRDLQIESLKREVEMLRSELEKIKLEAQRYIAQLKSQVNALEGELEEQRKQKQKALVDNEQLRHELAQLRAAQLEGERSQGLREEAERKASATEARYNKLKEKHSELVHVHAELLRKNADTAKQLTVTQQSQEEVARVKEQLAFQVEQVKRESELKLEEKSDQLEKLKRELEAKAGELARAQEALSHTEQSKSELSSRLDTLSAEKDALSGAVRQREADLLAAQSLVRETEAALSREQQRSSQEQGELQGRLAERESQEQGLRQRLLDEQFAVLRGAAAEAAGILQDAVSKLDDPLHLRCTSSPDYLVSRAQEALDAVSTLEEGHAQYLTSLADASALVAALTRFSHLAADTIINGGATSHLAPTDPADRLIDTCRECGARALELMGQLQDQQALRHMQASLVRTPLQGILQLGQELKPKSLDVRQEELGAVVDKEMAATSAAIEDAVRRIEDMMNQARHASSGVKLEVNERILNSCTDLMKAIRLLVTTSTSLQKEIVESGRGAATQQEFYAKNSRWTEGLISASKAVGWGATQLVEAADKVVLHTGKYEELIVCSHEIAASTAQLVAASKVKANKHSPHLSRLQECSRTVNERAANVVASTKSGQEQIEDRDTMDFSGLSLIKLKKQEMETQVRVLELEKTLEAERMRLGELRKQHYVLAGASGSPGEEVAIRPSTAPRSVTTKKPPLAQKPSVAPRQDHQLDKKDGIYPAQLVNY",
# RFIP3_HUMAN
"MASAPPASPPGSEPPGPDPEPGGPDGPGAAQLAPGPAELRLGAPVGGPDPQSPGLDEPAPGAAADGGARWSAGPAPGLEGGPRDPGPSAPPPRSGPRGQLASPDAPGPGPRSEAPLPELDPLFSWTEEPEECGPASCPESAPFRLQGSSSSHRARGEVDVFSPFPAPTAGELALEQGPGSPPQPSDLSQTHPLPSEPVGSQEDGPRLRAVFDALDGDGDGFVRIEDFIQFATVYGAEQVKDLTKYLDPSGLGVISFEDFYQGITAIRNGDPDGQCYGGVASAQDEEPLACPDEFDDFVTYEANEVTDSAYMGSESTYSECETFTDEDTSTLVHPELQPEGDADSAGGSAVPSECLDAMEEPDHGALLLLPGRPHPHGQSVITVIGGEEHFEDYGEGSEAELSPETLCNGQLGCSDPAFLTPSPTKRLSSKKVARYLHQSGALTMEALEDPSPELMEGPEEDIADKVVFLERRVLELEKDTAATGEQHSRLRQENLQLVHRANALEEQLKEQELRACEMVLEETRRQKELLCKMEREKSIEIENLQTRLQQLDEENSELRSCTPCLKANIERLEEEKQKLLDEIESLTLRLSEEQENKRRMGDRLSHERHQFQRDKEATQELIEDLRKQLEHLQLLKLEAEQRRGRSSSMGLQEYHSRARESELEQEVRRLKQDNRNLKEQNEELNGQIITLSIQGAKSLFSTAFSESLAAEISSVSRDELMEAIQKQEEINFRLQDYIDRIIVAIMETNPSILEVK",
# GLRA3_HUMAN
"MAHVRHFRTLVSGFYFWEAALLLSLVATKETDSARSRSAPMSPSDFLDKLMGRTSGYDARIRPNFKGPPVNVTCNIFINSFGSIAETTMDYRVNIFLRQKWNDPRLAYSEYPDDSLDLDPSMLDSIWKPDLFFANEKGANFHEVTTDNKLLRIFKNGNVLYSIRLTLTLSCPMDLKNFPMDVQTCIMQLESFGYTMNDLIFEWQDEAPVQVAEGLTLPQFLLKEEKDLRYCTKHYNTGKFTCIEVRFHLERQMGYYLIQMYIPSLLIVILSWVSFWINMDAAPARVALGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFSALLEYAAVNFVSRQHKELLRFRRKRKNKTEAFALEKFYRFSDMDDEVRESRFSFTAYGMGPCLQAKDGMTPKGPNHPVQVMPKSPDEMRKVFIDRAKKIDTISRACFPLAFLIFNIFYWVIYKILRHEDIHQQQD",
# SEM7A_HUMAN
"MTPPPPGRAAPSAPRARVPGPPARLGLPLRLRLLLLLWAAAASAQGHLRSGPRIFAVWKGHVGQDRVDFGQTEPHTVLFHEPGSSSVWVGGRGKVYLFDFPEGKNASVRTVNIGSTKGSCLDKRDCENYITLLERRSEGLLACGTNARHPSCWNLVNGTVVPLGEMRGYAPFSPDENSLVLFEGDEVYSTIRKQEYNGKIPRFRRIRGESELYTSDTVMQNPQFIKATIVHQDQAYDDKIYYFFREDNPDKNPEAPLNVSRVAQLCRGDQGGESSLSVSKWNTFLKAMLVCSDAATNKNFNRLQDVFLLPDPSGQWRDTRVYGVFSNPWNYSAVCVYSLGDIDKVFRTSSLKGYHSSLPNPRPGKCLPDQQPIPTETFQVADRHPEVAQRVEPMGPLKTPLFHSKYHYQKVAVHRMQASHGETFHVLYLTTDRGTIHKVVEPGEQEHSFAFNIMEIQPFRRAAAIQTMSLDAERRKLYVSSQWEVSQVPLDLCEVYGGGCHGCLMSRDPYCGWDQGRCISIYSSERSVLQSINPAEPHKECPNPKPDKAPLQKVSLAPNSRYYLSCPMESRHATYSWRHKENVEQSCEPGHQSPNCILFIENLTAQQYGHYFCEAQEGSYFREAQHWQLLPEDGIMAEHLLGHACALAASLWLGVLPTLTLGLLVH",
# LIPA2_HUMAN
"MMCEVMPTINEDTPMSQRGSQSSGSDSDSHFEQLMVNMLDERDRLLDTLRETQESLSLAQQRLQDVIYDRDSLQRQLNSALPQDIESLTGGLAGSKGADPPEFAALTKELNACREQLLEKEEEISELKAERNNTRLLLEHLECLVSRHERSLRMTVVKRQAQSPSGVSSEVEVLKALKSLFEHHKALDEKVRERLRVSLERVSALEEELAAANQEIVALREQNVHIQRKMASSEGSTESEHLEGMEPGQKVHEKRLSNGSIDSTDETSQIVELQELLEKQNYEMAQMKERLAALSSRVGEVEQEAETARKDLIKTEEMNTKYQRDIREAMAQKEDMEERITTLEKRYLSAQRESTSIHDMNDKLENELANKEAILRQMEEKNRQLQERLELAEQKLQQTMRKAETLPEVEAELAQRIAALTKAEERHGNIEERMRHLEGQLEEKNQELQRARQREKMNEEHNKRLSDTVDRLLTESNERLQLHLKERMAALEEKNVLIQESETFRKNLEESLHDKERLAEEIEKLRSELDQLKMRTGSLIEPTIPRTHLDTSAELRYSVGSLVDSQSDYRTTKVIRRPRRGRMGVRRDEPKVKSLGDHEWNRTQQIGVLSSHPFESDTEMSDIDDDDRETIFSSMDLLSPSGHSDAQTLAMMLQEQLDAINKEIRLIQEEKESTELRAEEIENRVASVSLEGLNLARVHPGTSITASVTASSLASSSPPSGHSTPKLTPRSPAREMDRMGVMTLPSDLRKHRRKIAVVEEDGREDKATIKCETSPPPTPRALRMTHTLPSSYHNDARSSLSVSLEPESLGLGSANSSQDSLHKAPKKKGIKSSIGRLFGKKEKARLGQLRGFMETEAAAQESLGLGKLGTQAEKDRRLKKKHELLEEARRKGLPFAQWDGPTVVAWLELWLGMPAWYVAACRANVKSGAIMSALSDTEIQREIGISNPLHRLKLRLAIQEMVSLTSPSAPPTSRTPSGNVWVTHEEMENLAAPAKTKESEEGSWAQCPVFLQTLAYGDMNHEWIGNEWLPSLGLPQYRSYFMECLVDARMLDHLTKKDLRVHLKMVDSFHRTSLQYGIMCLKRLNYDRKELERRREASQHEIKDVLVWSNDRVIRWIQAIGLREYANNILESGVHGSLIALDENFDYSSLALLLQIPTQNTQARQILEREYNNLLALGTERRLDESDDKNFRRGSTWRRQFPPREVHGISMMPGSSETLPAGFRLTTTSGQSRKMTTDVASSRLQRLDNSTVRTYSC",
# LIPA4_HUMAN
"MCEVMPTINEGDRLGPPHGADADANFEQLMVNMLDEREKLLESLRESQETLAATQSRLQDAIHERDQLQRHLNSALPQEFATLTRELSMCREQLLEREEEISELKAERNNTRLLLEHLECLVSRHERSLRMTVVKRQAQSPSGVSSEVEVLKALKSLFEHHKALDEKVRERLRAALERVTTLEEQLAGAHQQVSALQQGAGVRDGAAEEEGTVELGPKRLWKEDTGRVEELQELLEKQNFELSQARERLVTLTTTVTELEEDLGTARRDLIKSEELSSKHQRDLREALAQKEDMEERITTLEKRYLAAQREATSIHDLNDKLENELANKESLHRQCEEKARHLQELLEVAEQKLQQTMRKAETLPEVEAELAQRIAALTKAEERHGNIEEHLRQLEGQLEEKNQELARTAVQVRQREKMNEDHNKRLSDTVDRLLSESNERLQLHLKERMAALEEKNTLIQELESSQRQIEEQHHHKGRLSEEIEKLRQEVDQLKGRGGPFVDGVHSRSHMGSAADVRFSLGTTTHAPPGVHRRYSALREESAKLALPLTVTLRSPTWMRMSQGVCCNLEYHSSGTLCGSSGPLPVPEMIQEEKESTELRAEEIETRVTSGSMEALNLKQLRKRGSIPTSLTALSLASASPPLSGRSTPKLTSRSAAQDLDRMGVMTLPSDLRKHRRKLLSPVSREENREDKATIKCETSPPSSPRTLRLEKLGHPALSQEEGKSALEDQGSNPSSSNSSQDSLHKGAKRKGIKSSIGRLFGKKEKGRLIQLSRDGATGHVLLTDSEFSMQEPMVPAKLGTQAEKDRRLKKKHQLLEDARRKGMPFAQWDGPTVVSWLELWVGMPAWYVAACRANVKSGAIMSALSDTEIQREIGISNALHRLKLRLAIQEMVSLTSPSAPPTSRTSSGNVWVTHEEMETLETSTKTDSEEGSWAQTLAYGDMNHEWIGNEWLPSLGLPQYRSYFMECLVDARMLDHLTKKDLRVHLKMVDSFHRTSLQYGIMCLKRLNYDRKELEKRREESQHEIKDVLVWTNDQVVHWVQSIGLRDYAGNLHESGVHGALLALDENFDHNTLALILQIPTQNTQARQVMEREFNNLLALGTDRKLDDGDDKVFRRAPSWRKRFRPREHHGRGGMLSASAETLPAGFRVSTLGTLQPPPAPPKKIMPEAHSHYLYGHMLSAFRD",
# VATG1_HUMAN
"MASQSQGIQQLLQAEKRAAEKVSEARKRKNRRLKQAKEEAQAEIEQYRLQREKEFKAKEAAALGSRGSCSTEVEKETQEKMTILQTYFRQNRDEVLDNLLAFVCDIRPEIHENYRING",
# BCAS1_HUMAN
"MGNQMSVPQRVEDQENEPEAETYQDNASALNGVPVVVSTHTVQHLEEVDLGISVKTDNVATSSPETTEISAVADANGKNLGKEAKPEAPAAKSRFFLMLSRPVPGRTGDQAADSSLGSVKLDVSSNKAPANKDPSESWTLPVAAGPGQDTDKTPGHAPAQDKVLSAARDPTLLPPETGGAGGEAPSKPKDSSFFDKFFKLDKGQEKVPGDSQQEAKRAEHQDKVDEVPGLSGQSDDVPAGKDIVDGKEKEGQELGTADCSVPGDPEGLETAKDDSQAAAIAENNNSIMSFFKTLVSPNKAETKKDPEDTGAEKSPTTSADLKSDKANFTSQETQGAGKNSKGCNPSGHTQSVTTPEPAKEGTKEKSGPTSLPLGKLFWKKSVKEDSVPTGAEENVVCESPVEIIKSKEVESALQTVDLNEGDAAPEPTEAKLKREESKPRTSLMAFLRQMSVKGDGGITHSEEINGKDSSCQTSDSTEKTITPPEPEPTGAPQKGKEGSSKDKKSAAEMNKQKSNKQEAKEPAQCTEQATVDTNSLQNGDKLQKRPEKRQQSLGGFFKGLGPKRMLDAQVQTDPVSIGPVGKSK",
# VAMP4_HUMAN
"MPPKFKRHLNDDDVTGSVKSERRNLLEDDSDEEEDFFLRGPSGPRFGPRNDKIKHVQNQVDEVIDVMQENITKVIERGERLDELQDKSESLSDNATAFSNRSKQLRRQMWWRGCKIKAIMALVAAILLLVIIILIVMKYRT",
# TRIM3_HUMAN
"MAKREDSPGPEVQPMDKQFLVCSICLDRYQCPKVLPCLHTFCERCLQNYIPAQSLTLSCPVCRQTSILPEQGVSALQNNFFISSLMEAMQQAPDGAHDPEDPHPLSVVAGRPLSCPNHEGKTMEFYCEACETAMCGECRAGEHREHGTVLLRDVVEQHKAALQRQLEAVRGRLPQLSAAIALVGGISQQLQERKAEALAQISAAFEDLEQALQQRKQALVSDLETICGAKQKVLQSQLDTLRQGQEHIGSSCSFAEQALRLGSAPEVLLVRKHMRERLAALAAQAFPERPHENAQLELVLEVDGLRRSVLNLGALLTTSATAHETVATGEGLRQALVGQPASLTVTTKDKDGRLVRTGSAELRAEITGPDGTRLPVPVVDHKNGTYELVYTARTEGELLLSVLLYGQPVRGSPFRVRALRPGDLPPSPDDVKRRVKSPGGPGSHVRQKAVRRPSSMYSTGGKRKDNPIEDELVFRVGSRGREKGEFTNLQGVSAASSGRIVVADSNNQCIQVFSNEGQFKFRFGVRGRSPGQLQRPTGVAVDTNGDIIVADYDNRWVSIFSPEGKFKTKIGAGRLMGPKGVAVDRNGHIIVVDNKSCCVFTFQPNGKLVGRFGGRGATDRHFAGPHFVAVNNKNEIVVTDFHNHSVKVYSADGEFLFKFGSHGEGNGQFNAPTGVAVDSNGNIIVADWGNSRIQVFDSSGSFLSYINTSAEPLYGPQGLALTSDGHVVVADAGNHCFKAYRYLQ",
# SC22B_HUMAN
"MVLLTMIARVADGLPLAASMQEDEQSGRDLQQYQSQAKQLFRKLNEQSPTRCTLEAGAMTFHYIIEQGVCYLVLCEAAFPKKLAFAYLEDLHSEFDEQHGKKVPTVSRPYSFIEFDTFIQKTKKLYIDSRARRNLGSINTELQDVQRIMVANIEEVLQRGEALSALDSKANNLSSLSKKYRQDAKYLNMRSTYAKLAAVAVFFIMLIVYVRFWWL",
# GPC4_HUMAN
"MARFGLPALLCTLAVLSAALLAAELKSKSCSEVRRLYVSKGFNKNDAPLHEINGDHLKICPQGSTCCSQEMEEKYSLQSKDDFKSVVSEQCNHLQAVFASRYKKFDEFFKELLENAEKSLNDMFVKTYGHLYMQNSELFKDLFVELKRYYVVGNVNLEEMLNDFWARLLERMFRLVNSQYHFTDEYLECVSKYTEQLKPFGDVPRKLKLQVTRAFVAARTFAQGLAVAGDVVSKVSVVNPTAQCTHALLKMIYCSHCRGLVTVKPCYNYCSNIMRGCLANQGDLDFEWNNFIDAMLMVAERLEGPFNIESVMDPIDVKISDAIMNMQDNSVQVSQKVFQGCGPPKPLPAGRISRSISESAFSARFRPHHPEERPTTAAGTSLDRLVTDVKEKLKQAKKFWSSLPSNVCNDERMAAGNGNEDDCWNGKGKSRYLFAVTGNGLANQGNNPEVQVDTSKPDILILRQIMALRVMTSKMKNAYNGNDVDFFDISDESSGEGSGSGCEYQQCPSEFDYNATDHAGKSANEKADSAGVRPGAQAYLLTVFCILFLVMQREWR",
# DAB1_HUMAN
"MSTETELQVAVKTSAKKDSRKKGQDRSEATLIKRFKGEGVRYKAKLIGIDEVSAARGDKLCQDSMMKLKGVVAGARSKGEHKQKIFLTISFGGIKIFDEKTGALQHHHAVHEISYIAKDITDHRAFGYVCGKEGNHRFVAIKTAQAAEPVILDLRDLFQLIYELKQREELEKKAQKDKQCEQAVYQTILEEDVEDPVYQYIVFEAGHEPIRDPETEENIYQVPTSQKKEGVYDVPKSQPVSNGYSFEDFEERFAAATPNRNLPTDFDEIFEATKAVTQLELFGDMSTPPDITSPPTPATPGDAFIPSSSQTLPASADVFSSVPFGTAAVPSGYVAMGAVLPSFWGQQPLVQQQMVMGAQPPVAQVMPGAQPIAWGQPGLFPATQQPWPTVAGQFPPAAFMPTQTVMPLPAAMFQGPLTPLATVPGTSDSTRSSPQTDKPRQKMGKETFKDFQMAQPPPVPSRKPDQPSLTCTSEAFSSYFNKVGVAQDTDDCDDFDISQLNLTPVTSTTPSTNSPPTPAPRQSSPSKSSASHASDPTTDDIFEEGFESPSKSEEQEAPDGSQASSNSDPFGEPSGEPSGDNISPQAGS",
# KS6A5_HUMAN
"MEEEGGSSGGAAGTSADGGDGGEQLLTVKHELRTANLTGHAEKVGIENFELLKVLGTGAYGKVFLVRKISGHDTGKLYAMKVLKKATIVQKAKTTEHTRTERQVLEHIRQSPFLVTLHYAFQTETKLHLILDYINGGELFTHLSQRERFTEHEVQIYVGEIVLALEHLHKLGIIYRDIKLENILLDSNGHVVLTDFGLSKEFVADETERAYSFCGTIEYMAPDIVRGGDSGHDKAVDWWSLGVLMYELLTGASPFTVDGEKNSQAEISRRILKSEPPYPQEMSALAKDLIQRLLMKDPKKRLGCGPRDADEIKEHLFFQKINWDDLAAKKVPAPFKPVIRDELDVSNFAEEFTEMDPTYSPAALPQSSEKLFQGYSFVAPSILFKRNAAVIDPLQFHMGVERPGVTNVARSAMMKDSPFYQHYDLDLKDKPLGEGSFSICRKCVHKKSNQAFAVKIISKRMEANTQKEITALKLCEGHPNIVKLHEVFHDQLHTFLVMELLNGGELFERIKKKKHFSETEASYIMRKLVSAVSHMHDVGVVHRDLKPENLLFTDENDNLEIKIIDFGFARLKPPDNQPLKTPCFTLHYAAPELLNQNGYDESCDLWSLGVILYTMLSGQVPFQSHDRSLTCTSAVEIMKKIKKGDFSFEGEAWKNVSQEAKDLIQGLLTVDPNKRLKMSGLRYNEWLQDGSQLSSNPLMTPDILGSSGAAVHTCVKATFHAFNKYKREGFCLQNVDKAPLAKRRKMKKTSTSTETRSSSSESSHSSSSHSHGKTTPTKTLQPSNPADSNNPETLFQFSDSVA",
# KS6A4_HUMAN
"MGDEDDDESCAVELRITEANLTGHEEKVSVENFELLKVLGTGAYGKVFLVRKAGGHDAGKLYAMKVLRKAALVQRAKTQEHTRTERSVLELVRQAPFLVTLHYAFQTDAKLHLILDYVSGGEMFTHLYQRQYFKEAEVRVYGGEIVLALEHLHKLGIIYRDLKLENVLLDSEGHIVLTDFGLSKEFLTEEKERTFSFCGTIEYMAPEIIRSKTGHGKAVDWWSLGILLFELLTGASPFTLEGERNTQAEVSRRILKCSPPFPPRIGPVAQDLLQRLLCKDPKKRLGAGPQGAQEVRNHPFFQGLDWVALAARKIPAPFRPQIRSELDVGNFAEEFTRLEPVYSPPGSPPPGDPRIFQGYSFVAPSILFDHNNAVMTDGLEAPGAGDRPGRAAVARSAMMQDSPFFQQYELDLREPALGQGSFSVCRRCRQRQSGQEFAVKILSRRLEANTQREVAALRLCQSHPNVVNLHEVHHDQLHTYLVLELLRGGELLEHIRKKRHFSESEASQILRSLVSAVSFMHEEAGVVHRDLKPENILYADDTPGAPVKIIDFGFARLRPQSPGVPMQTPCFTLQYAAPELLAQQGYDESCDLWSLGVILYMMLSGQVPFQGASGQGGQSQAAEIMCKIREGRFSLDGEAWQGVSEEAKELVRGLLTVDPAKRLKLEGLRGSSWLQDGSARSSPPLRTPDVLESSGPAVRSGLNATFMAFNRGKREGFFLKSVENAPLAKRRKQKLRSATASRRGSPAPANPGRAPVASKGAPRRANGPLPPS",
# PALM_HUMAN
"MEVLAAETTSQQERLQAIAEKRKRQAEIENKRRQLEDERRQLQHLKSKALRERWLLEGTPSSASEGDEDLRRQMQDDEQKTRLLEDSVSRLEKEIEVLERGDSAPATAKENAAAPSPVRAPAPSPAKEERKTEVVMNSQQTPVGTPKDKRVSNTPLRTVDGSPMMKAAMYSVEITVEKDKVTGETRVLSSTTLLPRQPLPLGIKVYEDETKVVHAVDGTAENGIHPLSSSEVDELIHKADEVTLSEAGSTAGAAETRGAVEGAARTTPSRREITGVQAQPGEATSGPPGIQPGQEPPVTMIFMGYQNVEDEAETKKVLGLQDTITAELVVIEDAAEPKEPAPPNGSAAEPPTEAASREENQAGPEATTSDPQDLDMKKHRCKCCSIM",
# RENR_HUMAN
"MAVFVVLLALVAGVLGNEFSILKSPGSVVFRNGNWPIPGERIPDVAALSMGFSVKEDLSWPGLAVGNLFHRPRATVMVMVKGVNKLALPPGSVISYPLENAVPFSLDSVANSIHSLFSEETPVVLQLAPSEERVYMVGKANSVFEDLSVTLRQLRNRLFQENSVLSSLPLNSLSRNNEVDLLFLSELQVLHDISSLLSRHKHLAKDHSPDLYSLELAGLDEIGKRYGEDSEQFRDASKILVDALQKFADDMYSLYGGNAVVELVTVKSFDTSLIRKTRTILEAKQAKNPASPYNLAYKYNFEYSVVFNMVLWIMIALALAVIITSYNIWNMDPGYDSIIYRMTNQKIRMD",
# AP1G2_HUMAN
"MVVPSLKLQDLIEEIRGAKTQAQEREVIQKECAHIRASFRDGDPVHRHRQLAKLLYVHMLGYPAHFGQMECLKLIASSRFTDKRVGYLGAMLLLDERHDAHLLITNSIKNDLSQGIQPVQGLALCTLSTMGSAEMCRDLAPEVEKLLLQPSPYVRKKAILTAVHMIRKVPELSSVFLPPCAQLLHERHHGILLGTITLITELCERSPAALRHFRKVVPQLVHILRTLVTMGYSTEHSISGVSDPFLQVQILRLLRILGRNHEESSETMNDLLAQVATNTDTSRNAGNAVLFETVLTIMDIRSAAGLRVLAVNILGRFLLNSDRNIRYVALTSLLRLVQSDHSAVQRHRPTVVECLRETDASLSRRALELSLALVNSSNVRAMMQELQAFLESCPPDLRADCASGILLAAERFAPTKRWHIDTILHVLTTAGTHVRDDAVANLTQLIGGAQELHAYSVRRLYNALAEDISQQPLVQVAAWCIGEYGDLLLAGNCEEIEPLQVDEEEVLALLEKVLQSHMSLPATRGYALTALMKLSTRLCGDNNRIRQVVSIYGSCLDVELQQRAVEYDTLFRKYDHMRAAILEKMPLVERDGPQADEEAKESKEAAQLSEAAPVPTEPQASQLLDLLDLLDGASGDVQHPPHLDPSPGGALVHLLDLPCVPPPPAPIPDLKVFEREGVQLNLSFIRPPENPALLLITITATNFSEGDVTHFICQAAVPKSLQLQLQAPSGNTVPARGGLPITQLFRILNPNKAPLRLKLRLTYDHFHQSVQEIFEVNNLPVESWQ",
# GABR2_HUMAN
"MASPRSSGQPGPPPPPPPPPARLLLLLLLPLLLPLAPGAWGWARGAPRPPPSSPPLSIMGLMPLTKEVAKGSIGRGVLPAVELAIEQIRNESLLRPYFLDLRLYDTECDNAKGLKAFYDAIKYGPNHLMVFGGVCPSVTSIIAESLQGWNLVQLSFAATTPVLADKKKYPYFFRTVPSDNAVNPAILKLLKHYQWKRVGTLTQDVQRFSEVRNDLTGVLYGEDIEISDTESFSNDPCTSVKKLKGNDVRIILGQFDQNMAAKVFCCAYEENMYGSKYQWIIPGWYEPSWWEQVHTEANSSRCLRKNLLAAMEGYIGVDFEPLSSKQIKTISGKTPQQYEREYNNKRSGVGPSKFHGYAYDGIWVIAKTLQRAMETLHASSRHQRIQDFNYTDHTLGRIILNAMNETNFFGVTGQVVFRNGERMGTIKFTQFQDSREVKVGEYNAVADTLEIINDTIRFQGSEPPKDKTIILEQLRKISLPLYSILSALTILGMIMASAFLFFNIKNRNQKLIKMSSPYMNNLIILGGMLSYASIFLFGLDGSFVSEKTFETLCTVRTWILTVGYTTAFGAMFAKTWRVHAIFKNVKMKKKIIKDQKLLVIVGGMLLIDLCILICWQAVDPLRRTVEKYSMEPDPAGRDISIRPLLEHCENTHMTIWLGIVYAYKGLLMLFGCFLAWETRNVSIPALNDSKYIGMSVYNVGIMCIIGAAVSFLTRDQPNVQFCIVALVIIFCSTITLCLVFVPKLITLRTNPDAATQNRRFQFTQNQKKEDSKTSTSVTSVNQASTSRLEGLQSENHRLRMKITELDKDLEEVTMQLQDTPEKTTYIKQNHYQELNDILNLGNFTESTDGGKAILKNHLDQNPQLQWNTTEPSRTCKDPIEDINSPEHIQRRLSLQLPILHHAYLPSIGGVDASCVSPCVSPTASPRHRHVPPSFRVMVSGL",
# DGKI_HUMAN
"MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSAAAGAGAMNPSSSAGEEKGATGGSSSSGSGAGSCCLGAEGGADPRGAGSAAAAGAAALDEPAAAGQKEKDEALEEKLRNLTFRKQVSYRKAISRAGLQHLAPAHPLSLPVANGPAKEPRATLDWSENAVNGEHLWLETNVSGDLCYLGEENCQVRFAKSALRRKCAVCKIVVHTACIEQLEKINFRCKPTFREGGSRSPRENFVRHHWVHRRRQEGKCKQCGKGFQQKFSFHSKEIVAISCSWCKQAFHNKVTCFMLHHIEEPCSLGAHAAVIVPPTWIIKVKKPQNSLKASNRKKKRTSFKRKASKRGMEQENKGRPFVIKPISSPLMKPLLVFVNPKSGGNQGTKVLQMFMWYLNPRQVFDLSQEGPKDALELYRKVPNLRILACGGDGTVGWILSILDELQLSPQPPVGVLPLGTGNDLARTLNWGGGYTDEPVSKILCQVEDGTVVQLDRWNLHVERNPDLPPEELEDGVCKLPLNVFNNYFSLGFDAHVTLEFHESREANPEKFNSRFRNKMFYAGAAFSDFLQRSSRDLSKHVKVVCDGTDLTPKIQELKFQCIVFLNIPRYCAGTMPWGNPGDHHDFEPQRHDDGYIEVIGFTMASLAALQVGGHGERLHQCREVMLLTYKSIPMQVDGEPCRLAPAMIRISLRNQANMVQKSKRRTSMPLLNDPQSVPDRLRIRVNKISLQDYEGFHYDKEKLREASIPLGILVVRGDCDLETCRMYIDRLQEDLQSVSSGSQRVHYQDHETSFPRALSAQRLSPRWCFLDATSADRFYRIDRSQEHLHFVMEISQDEIFILDPDMVVSQPAGTPPGMPDLVVEQASGISDWWNPALRKRMLSDSGLGMIAPYYEDSDLKDLSHSRVLQSPVSSEDHAILQAVIAGDLMKLIESYKNGGSLLIQGPDHCSLLHYAAKTGNGEIVKYILDHGPSELLDMADSETGETALHKAACQRNRAVCQLLVDAGASLRKTDSKGKTPQERAQQAGDPDLAAYLESRQNYKVIGHEDLETAV",
# PAK3_HUMAN
"MSDGLDNEEKPPAPPLRMNSNNRDSSALNHSSKPLPMAPEEKNKKARLRSIFPGGGDKTNKKKEKERPEISLPSDFEHTIHVGFDAVTGEFTPDLYGSQMCPGKLPEGIPEQWARLLQTSNITKLEQKKNPQAVLDVLKFYDSKETVNNQKYMSFTSGDKSAHGYIAAHPSSTKTASEPPLAPPVSEEEDEEEEEEEDENEPPPVIAPRPEHTKSIYTRSVVESIASPAVPNKEVTPPSAENANSSTLYRNTDRQRKKSKMTDEEILEKLRSIVSVGDPKKKYTRFEKIGQGASGTVYTALDIATGQEVAIKQMNLQQQPKKELIINEILVMRENKNPNIVNYLDSYLVGDELWVVMEYLAGGSLTDVVTETCMDEGQIAAVCRECLQALDFLHSNQVIHRDIKSDNILLGMDGSVKLTDFGFCAQITPEQSKRSTMVGTPYWMAPEVVTRKAYGPKVDIWSLGIMAIEMVEGEPPYLNENPLRALYLIATNGTPELQNPERLSAVFRDFLNRCLEMDVDRRGSAKELLQHPFLKLAKPLSSLTPLIIAAKEAIKNSSR",
# PRAF3_HUMAN
"MDVNIAPLRAWDDFFPGSDRFARPDFRDISKWNNRVVSNLLYYQTNYLVVAAMMISIVGFLSPFNMILGGIVVVLVFTGFVWAAHNKDVLRRMKKRYPTTFVMVVMLASYFLISMFGGVMVFVFGITFPLLLMFIHASLRLRNLKNKLENKMEGIGLKRTPMGIVLDALEQQEEGINRLTDYISKVKE",
# RGS9_HUMAN
"MTIRHQGQQYRPRMAFLQKIEALVKDMQNPETGVRMQNQRVLVTSVPHAMTGSDVLQWIVQRLWISSLEAQNLGNFIVRYGYIYPLQDPKNLILKPDGSLYRFQTPYFWPTQQWPAEDTDYAIYLAKRNIKKKGILEEYEKENYNFLNQKMNYKWDFVIMQAKEQYRAGKERNKADRYALDCQEKAYWLVHRCPPGMDNVLDYGLDRVTNPNEVKVNQKQTVVAVKKEIMYYQQALMRSTVKSSVSLGGIVKYSEQFSSNDAIMSGCLPSNPWITDDTQFWDLNAKLVEIPTKMRVERWAFNFSELIRDPKGRQSFQYFLKKEFSGENLGFWEACEDLKYGDQSKVKEKAEEIYKLFLAPGARRWINIDGKTMDITVKGLKHPHRYVLDAAQTHIYMLMKKDSYARYLKSPIYKDMLAKAIEPQETTKKSSTLPFMRRHLRSSPSPVILRQLEEEAKAREAANTVDITQPGQHMAPSPHLTVYTGTCMPPSPSSPFSSSCRSPRKPFASPSRFIRRPSTTICPSPIRVALESSSGLEQKGECSGSMAPRGPSVTESSEASLDTSWPRSRPRAPPKARMALSFSRFLRRGCLASPVFARLSPKCPAVSHGRVQPLGDVGQQLPRLKSKRVANFFQIKMDVPTGSGTCLMDSEDAGTGESGDRATEKEVICPWESL",
# PIAS1_HUMAN
"MADSAELKQMVMSLRVSELQVLLGYAGRNKHGRKHELLTKALHLLKAGCSPAVQMKIKELYRRRFPQKIMTPADLSIPNVHSSPMPATLSPSTIPQLTYDGHPASSPLLPVSLLGPKHELELPHLTSALHPVHPDIKLQKLPFYDLLDELIKPTSLASDNSQRFRETCFAFALTPQQVQQISSSMDISGTKCDFTVQVQLRFCLSETSCPQEDHFPPNLCVKVNTKPCSLPGYLPPTKNGVEPKRPSRPINITSLVRLSTTVPNTIVVSWTAEIGRNYSMAVYLVKQLSSTVLLQRLRAKGIRNPDHSRALIKEKLTADPDSEIATTSLRVSLLCPLGKMRLTIPCRALTCSHLQCFDATLYIQMNEKKPTWVCPVCDKKAPYEHLIIDGLFMEILKYCTDCDEIQFKEDGTWAPMRSKKEVQEVSASYNGVDGCLSSTLEHQVASHHQSSNKNKKVEVIDLTIDSSSDEEEEEPSAKRTCPSLSPTSPLNNKGILSLPHQASPVSRTPSLPAVDTSYINTSLIQDYRHPFHMTPMPYDLQGLDFFPFLSGDNQHYNTSLLAAAAAAVSDDQDLLHSSRFFPYTSSQMFLDQLSAGGSTSLPTTNGSSSGSNSSLVSSNSLRESHSHTVTNRSSTDTASIFGIIPDIISLD",
# FLOT1_HUMAN
"MFFTCGPNEAMVVSGFCRSPPVMVAGGRVFVLPCIQQIQRISLNTLTLNVKSEKVYTRHGVPISVTGIAQVKIQGQNKEMLAAACQMFLGKTEAEIAHIALETLEGHQRAIMAHMTVEEIYKDRQKFSEQVFKVASSDLVNMGISVVSYTLKDIHDDQDYLHSLGKARTAQVQKDARIGEAEAKRDAGIREAKAKQEKVSAQYLSEIEMAKAQRDYELKKAAYDIEVNTRRAQADLAYQLQVAKTKQQIEEQRVQVQVVERAQQVAVQEQEIARREKELEARVRKPAEAERYKLERLAEAEKSQLIMQAEAEAASVRMRGEAEAFAIGARARAEAEQMAKKAEAFQLYQEAAQLDMLLEKLPQVAEEISGPLTSANKITLVSSGSGTMGAAKVTGEVLDILTRLPESVERLTGVSISQVNHKPLRTA",
# TRIO_HUMAN
"MSGSSGGAAAPAASSGPAAAASAAGSGCGGGAGEGAEEAAKDLADIAAFFRSGFRKNDEMKAMDVLPILKEKVAYLSGGRDKRGGPILTFPARSNHDRIRQEDLRRLISYLACIPSEEVCKRGFTVIVDMRGSKWDSIKPLLKILQESFPCCIHVALIIKPDNFWQKQRTNFGSSKFEFETNMVSLEGLTKVVDPSQLTPEFDGCLEYNHEEWIEIRVAFEDYISNATHMLSRLEELQDILAKKELPQDLEGARNMIEEHSQLKKKVIKAPIEDLDLEGQKLLQRIQSSESFPKKNSGSGNADLQNLLPKVSTMLDRLHSTRQHLHQMWHVRKLKLDQCFQLRLFEQDAEKMFDWITHNKGLFLNSYTEIGTSHPHAMELQTQHNHFAMNCMNVYVNINRIMSVANRLVESGHYASQQIRQIASQLEQEWKAFAAALDERSTLLDMSSIFHQKAEKYMSNVDSWCKACGEVDLPSELQDLEDAIHHHQGIYEHITLAYSEVSQDGKSLLDKLQRPLTPGSSDSLTASANYSKAVHHVLDVIHEVLHHQRQLENIWQHRKVRLHQRLQLCVFQQDVQQVLDWIENHGEAFLSKHTGVGKSLHRARALQKRHEDFEEVAQNTYTNADKLLEAAEQLAQTGECDPEEIYQAAHQLEDRIQDFVRRVEQRKILLDMSVSFHTHVKELWTWLEELQKELLDDVYAESVEAVQDLIKRFGQQQQTTLQVTVNVIKEGEDLIQQLRDSAISSNKTPHNSSINHIETVLQQLDEAQSQMEELFQERKIKLELFLQLRIFERDAIDIISDLESWNDELSQQMNDFDTEDLTIAEQRLQHHADKALTMNNLTFDVIHQGQDLLQYVNEVQASGVELLCDRDVDMATRVQDLLEFLHEKQQELDLAAEQHRKHLEQCVQLRHLQAEVKQVLGWIRNGESMLNAGLITASSLQEAEQLQREHEQFQHAIEKTHQSALQVQQKAEAMLQANHYDMDMIRDCAEKVASHWQQLMLKMEDRLKLVNASVAFYKTSEQVCSVLESLEQEYKREEDWCGGADKLGPNSETDHVTPMISKHLEQKEAFLKACTLARRNADVFLKYLHRNSVNMPGMVTHIKAPEQQVKNILNELFQRENRVLHYWTMRKRRLDQCQQYVVFERSAKQALEWIHDNGEFYLSTHTSTGSSIQHTQELLKEHEEFQITAKQTKERVKLLIQLADGFCEKGHAHAAEIKKCVTAVDKRYRDFSLRMEKYRTSLEKALGISSDSNKSSKSLQLDIIPASIPGSEVKLRDAAHELNEEKRKSARRKEFIMAELIQTEKAYVRDLRECMDTYLWEMTSGVEEIPPGIVNKELIIFGNMQEIYEFHNNIFLKELEKYEQLPEDVGHCFVTWADKFQMYVTYCKNKPDSTQLILEHAGSYFDEIQQRHGLANSISSYLIKPVQRITKYQLLLKELLTCCEEGKGEIKDGLEVMLSVPKRANDAMHLSMLEGFDENIESQGELILQESFQVWDPKTLIRKGRERHLFLFEMSLVFSKEVKDSSGRSKYLYKSKLFTSELGVTEHVEGDPCKFALWVGRTPTSDNKIVLKASSIENKQDWIKHIREVIQERTIHLKGALKEPIHIPKTAPATRQKGRRDGEDLDSQGDGSSQPDTISIASRTSQNTLDSDKLSGGCELTVVIHDFTACNSNELTIRRGQTVEVLERPHDKPDWCLVRTTDRSPAAEGLVPCGSLCIAHSRSSMEMEGIFNHKDSLSVSSNDASPPASVASLQPHMIGAQSSPGPKRPGNTLRKWLTSPVRRLSSGKADGHVKKLAHKHKKSREVRKSADAGSQKDSDDSAATPQDETVEERGRNEGLSSGTLSKSSSSGMQSCGEEEGEEGADAVPLPPPMAIQQHSLLQPDSQDDKASSRLLVRPTSSETPSAAELVSAIEELVKSKMALEDRPSSLLVDQGDSSSPSFNPSDNSLLSSSSPIDEMEERKSSSLKRRHYVLQELVETERDYVRDLGYVVEGYMALMKEDGVPDDMKGKDKIVFGNIHQIYDWHRDFFLGELEKCLEDPEKLGSLFVKHERRLHMYIAYCQNKPKSEHIVSEYIDTFFEDLKQRLGHRLQLTDLLIKPVQRIMKYQLLLKDFLKYSKKASLDTSELERAVEVMCIVPRRCNDMMNVGRLQGFDGKIVAQGKLLLQDTFLVTDQDAGLLPRCRERRIFLFEQIVIFSEPLDKKKGFSMPGFLFKNSIKVSCLCLEENVENDPCKFALTSRTGDVVETFILHSSSPSVRQTWIHEINQILENQRNFLNALTSPIEYQRNHSGGGGGGGSGGSGGGGGSGGGGAPSGGSGHSGGPSSCGGAPSTSRSRPSRIPQPVRHHPPVLVSSAASSQAEADKMSGTSTPGPSLPPPGAAPEAGPSAPSRRPPGADAEGSEREAEPIPKMKVLESPRKGAANASGSSPDAPAKDARASLGTLPLGKPRAGAASPLNSPLSSAVPSLGKEPFPPSSPLQKGGSFWSSIPASPASRPGSFTFPGDSDSLQRQTPRHAAPGKDTDRMSTCSSASEQSVQSTQSNGSESSSSSNISTMLVTHDYTAVKEDEINVYQGEVVQILASNQQNMFLVFRAATDQCPAAEGWIPGFVLGHTSAVIVENPDGTLKKSTSWHTALRLRKKSEKKDKDGKREGKLENGYRKSREGLSNKVSVKLLNPNYIYDVPPEFVIPLSEVTCETGETVVLRCRVCGRPKASITWKGPEHNTLNNDGHYSISYSDLGEATLKIVGVTTEDDGIYTCIAVNDMGSASSSASLRVLGPGMDGIMVTWKDNFDSFYSEVAELGRGRFSVVKKCDQKGTKRAVATKFVNKKLMKRDQVTHELGILQSLQHPLLVGLLDTFETPTSYILVLEMADQGRLLDCVVRWGSLTEGKIRAHLGEVLEAVRYLHNCRIAHLDLKPENILVDESLAKPTIKLADFGDAVQLNTTYYIHQLLGNPEFAAPEIILGNPVSLTSDTWSVGVLTYVLLSGVSPFLDDSVEETCLNICRLDFSFPDDYFKGVSQKAKEFVCFLLQEDPAKRPSAALALQEQWLQAGNGRSTGVLDTSRLTSFIERRKHQNDVRPIRSIKNFLQSRLLPRV",
# MPDZ_HUMAN
"MLEAIDKNRALHAAERLQTKLRERGDVANEDKLSLLKSVLQSPLFSQILSLQTSVQQLKDQVNIATSATSNIEYAHVPHLSPAVIPTLQNESFLLSPNNGNLEALTGPGIPHINGKPACDEFDQLIKNMAQGRHVEVFELLKPPSGGLGFSVVGLRSENRGELGIFVQEIQEGSVAHRDGRLKETDQILAINGQALDQTITHQQAISILQKAKDTVQLVIARGSLPQLVSPIVSRSPSAASTISAHSNPVHWQHMETIELVNDGSGLGFGIIGGKATGVIVKTILPGGVADQHGRLCSGDHILKIGDTDLAGMSSEQVAQVLRQCGNRVKLMIARGAIEERTAPTALGITLSSSPTSTPELRVDASTQKGEESETFDVELTKNVQGLGITIAGYIGDKKLEPSGIFVKSITKSSAVEHDGRIQIGDQIIAVDGTNLQGFTNQQAVEVLRHTGQTVLLTLMRRGMKQEAELMSREDVTKDADLSPVNASIIKENYEKDEDFLSSTRNTNILPTEEEGYPLLSAEIEEIEDAQKQEAALLTKWQRIMGINYEIVVAHVSKFSENSGLGISLEATVGHHFIRSVLPEGPVGHSGKLFSGDELLEVNGITLLGENHQDVVNILKELPIEVTMVCCRRTVPPTTQSELDSLDLCDIELTEKPHVDLGEFIGSSETEDPVLAMTDAGQSTEEVQAPLAMWEAGIQHIELEKGSKGLGFSILDYQDPIDPASTVIIIRSLVPGGIAEKDGRLLPGDRLMFVNDVNLENSSLEEAVEALKGAPSGTVRIGVAKPLPLSPEEGYVSAKEDSFLYPPHSCEEAGLADKPLFRADLALVGTNDADLVDESTFESPYSPENDSIYSTQASILSLHGSSCGDGLNYGSSLPSSPPKDVIENSCDPVLDLHMSLEELYTQNLLQRQDENTPSVDISMGPASGFTINDYTPANAIEQQYECENTIVWTESHLPSEVISSAELPSVLPDSAGKGSEYLLEQSSLACNAECVMLQNVSKESFERTINIAKGNSSLGMTVSANKDGLGMIVRSIIHGGAISRDGRIAIGDCILSINEESTISVTNAQARAMLRRHSLIGPDIKITYVPAEHLEEFKISLGQQSGRVMALDIFSSYTGRDIPELPEREEGEGEESELQNTAYSNWNQPRRVELWREPSKSLGISIVGGRGMGSRLSNGEVMRGIFIKHVLEDSPAGKNGTLKPGDRIVEVDGMDLRDASHEQAVEAIRKAGNPVVFMVQSIINRPRKSPLPSLLHNLYPKYNFSSTNPFADSLQINADKAPSQSESEPEKAPLCSVPPPPPSAFAEMGSDHTQSSASKISQDVDKEDEFGYSWKNIRERYGTLTGELHMIELEKGHSGLGLSLAGNKDRSRMSVFIVGIDPNGAAGKDGRLQIADELLEINGQILYGRSHQNASSIIKCAPSKVKIIFIRNKDAVNQMAVCPGNAVEPLPSNSENLQNKETEPTVTTSDAAVDLSSFKNVQHLELPKDQGGLGIAISEEDTLSGVIIKSLTEHGVAATDGRLKVGDQILAVDDEIVVGYPIEKFISLLKTAKMTVKLTIHAENPDSQAVPSAAGAASGEKKNSSQSLMVPQSGSPEPESIRNTSRSSTPAIFASDPATCPIIPGCETTIEISKGRTGLGLSIVGGSDTLLGAIIIHEVYEEGAACKDGRLWAGDQILEVNGIDLRKATHDEAINVLRQTPQRVRLTLYRDEAPYKEEEVCDTLTIELQKKPGKGLGLSIVGKRNDTGVFVSDIVKGGIADADGRLMQGDQILMVNGEDVRNATQEAVAALLKCSLGTVTLEVGRIKAGPFHSERRPSQSSQVSEGSLSSFTFPLSGSSTSESLESSSKKNALASEIQGLRTVEMKKGPTDSLGISIAGGVGSPLGDVPIFIAMMHPTGVAAQTQKLRVGDRIVTICGTSTEGMTHTQAVNLLKNASGSIEMQVVAGGDVSVVTGHQQEPASSSLSFTGLTSSSIFQDDLGPPQCKSITLERGPDGLGFSIVGGYGSPHGDLPIYVKTVFAKGAASEDGRLKRGDQIIAVNGQSLEGVTHEEAVAILKRTKGTVTLMVLS",
# C1QRF_HUMAN
"MLLVLVVLIPVLVSSGGPEGHYEMLGTCRMVCDPYPARGPGAGARTDGGDALSEQSGAPPPSTLVQGPQGKPGRTGKPGPPGPPGDPGPPGPVGPPGEKGEPGKPGPPGLPGAGGSGAISTATYTTVPRVAFYAGLKNPHEGYEVLKFDDVVTNLGNNYDAASGKFTCNIPGTYFFTYHVLMRGGDGTSMWADLCKNGQVRASAIAQDADQNYDYASNSVILHLDAGDEVFIKLDGGKAHGGNSNKYSTFSGFIIYSD",
# WFS1_HUMAN
"MDSNTAPLGPSCPQPPPAPQPQARSRLNATASLEQERSERPRAPGPQAGPGPGVRDAAAPAEPQAQHTRSRERADGTGPTKGDMEIPFEEVLERAKAGDPKAQTEVGKHYLQLAGDTDEELNSCTAVDWLVLAAKQGRREAVKLLRRCLADRRGITSENEREVRQLSSETDLERAVRKAALVMYWKLNPKKKKQVAVAELLENVGQVNEHDGGAQPGPVPKSLQKQRRMLERLVSSESKNYIALDDFVEITKKYAKGVIPSSLFLQDDEDDDELAGKSPEDLPLRLKVVKYPLHAIMEIKEYLIDMASRAGMHWLSTIIPTHHINALIFFFIVSNLTIDFFAFFIPLVIFYLSFISMVICTLKVFQDSKAWENFRTLTDLLLRFEPNLDVEQAEVNFGWNHLEPYAHFLLSVFFVIFSFPIASKDCIPCSELAVITGFFTVTSYLSLSTHAEPYTRRALATEVTAGLLSLLPSMPLNWPYLKVLGQTFITVPVGHLVVLNVSVPCLLYVYLLYLFFRMAQLRNFKGTYCYLVPYLVCFMWCELSVVILLESTGLGLLRASIGYFLFLFALPILVAGLALVGVLQFARWFTSLELTKIAVTVAVCSVPLLLRWWTKASFSVVGMVKSLTRSSMVKLILVWLTAIVLFCWFYVYRSEGMKVYNSTLTWQQYGALCGPRAWKETNMARTQILCSHLEGHRVTWTGRFKYVRVTDIDNSAESAINMLPFFIGDWMRCLYGEAYPACSPGNTSTAEEELCRLKLLAKHPCHIKKFDRYKFEITVGMPFSSGADGSRSREEDDVTKDIVLRASSEFKSVLLSLRQGSLIEFSTILEGRLGSKWPVFELKAISCLNCMAQLSPTRRHVKIEHDWRSTVHGAVKFAFDFFFFPFLSAA",
# CDKL5_HUMAN
"MKIPNIGNVMNKFEILGVVGEGAYGVVLKCRHKETHEIVAIKKFKDSEENEEVKETTLRELKMLRTLKQENIVELKEAFRRRGKLYLVFEYVEKNMLELLEEMPNGVPPEKVKSYIYQLIKAIHWCHKNDIVHRDIKPENLLISHNDVLKLCDFGFARNLSEGNNANYTEYVATRWYRSPELLLGAPYGKSVDMWSVGCILGELSDGQPLFPGESEIDQLFTIQKVLGPLPSEQMKLFYSNPRFHGLRFPAVNHPQSLERRYLGILNSVLLDLMKNLLKLDPADRYLTEQCLNHPTFQTQRLLDRSPSRSAKRKPYHVESSTLSNRNQAGKSTALQSHHRSNSKDIQNLSVGLPRADEGLPANESFLNGNLAGASLSPLHTKTYQASSQPGSTSKDLTNNNIPHLLSPKEAKSKTEFDFNIDPKPSEGPGTKYLKSNSRSQQNRHSFMESSQSKAGTLQPNEKQSRHSYIDTIPQSSRSPSYRTKAKSHGALSDSKSVSNLSEARAQIAEPSTSRYFPSSCLDLNSPTSPTPTRHSDTRTLLSPSGRNNRNEGTLDSRRTTTRHSKTMEELKLPEHMDSSHSHSLSAPHESFSYGLGYTSPFSSQQRPHRHSMYVTRDKVRAKGLDGSLSIGQGMAARANSLQLLSPQPGEQLPPEMTVARSSVKETSREGTSSFHTRQKSEGGVYHDPHSDDGTAPKENRHLYNDPVPRRVGSFYRVPSPRPDNSFHENNVSTRVSSLPSESSSGTNHSKRQPAFDPWKSPENISHSEQLKEKEKQGFFRSMKKKKKKSQTVPNSDSPDLLTLQKSIHSASTPSSRPKEWRPEKISDLQTQSQPLKSLRKLLHLSSASNHPASSDPRFQPLTAQQTKNSFSEIRIHPLSQASGGSSNIRQEPAPKGRPALQLPGQMDPGWHVSSVTRSATEGPSYSEQLGAKSGPNGHPYNRTNRSRMPNLNDLKETAL",
# NEUL1_HUMAN
"MGNNFSSIPSLPRGNPSRAPRGHPQNLKDSIGGPFPVTSHRCHHKQKHCPAVLPSGGLPATPLLFHPHTKGSQILMDLSHKAVKRQASFCNAITFSNRPVLIYEQVRLKITKKQCCWSGALRLGFTSKDPSRIHPDSLPKYACPDLVSQSGFWAKALPEEFANEGNIIAFWVDKKGRVFHRINDSAVMLFFSGVRTADPLWALVDVYGLTRGVQLLDSELVLPDCLRPRSFTALRRPSLRREADDARLSVSLCDLNVPGADGDEAAPAAGCPIPQNSLNSQHSRALPAQLDGDLRFHALRAGAHVRILDEQTVARVEHGRDERALVFTSRPVRVAETIFVKVTRSGGARPGALSFGVTTCDPGTLRPADLPFSPEALVDRKEFWAVCRVPGPLHSGDILGLVVNADGELHLSHNGAAAGMQLCVDASQPLWMLFGLHGTITQIRILGSTILAERGIPSLPCSPASTPTSPSALGSRLSDPLLSTCSSGPLGSSAGGTAPNSPVSLPESPVTPGLGQWSDECTICYEHAVDTVIYTCGHMCLCYACGLRLKKALHACCPICRRPIKDIIKTYRSS",
# CNTN5_HUMAN
"MASSWKLMLFLSVTMCLSEYSKSLPGLSTSYAALLRIKKSSSSSLFGSKTRPRYSSPSLGTLSASSPSWLGAAQNYYSPINLYHSSDAFKQDESVDYGPVFVQEPDDIIFPTDSDEKKVALNCEVRGNPVPSYRWLRNGTEIDLESDYRYSLIDGTFIISNPSEAKDSGHYQCLATNTVGSILSREATLQFAYLGNFSGRTRSAVSVREGQGVVLMCSPPPHSPEIIYSWVFNEFPSFVAEDSRRFISQETGNLYISKVQTSDVGSYICLVKNTVTNARVLSPPTPLTLRNDGVMGEYEPKIEVHFPFTVTAAKGTTVKMECFALGNPVPTITWMKVNGYIPSKARLRKSQAVLEIPNVQLDDAGIYECRAENSRGKNSFRGQLQVYTYPHWVEKLNDTQLDSGSPLRWECKATGKPRPTYRWLKNGVPLSPQSRVEMVNGVLMIHNVNQSDAGMYQCLAENKYGAIYASAELKILASAPTFALNQLKKTIIVTKDQEVVIECKPQGSPKPTISWKKGDRAVRENKRIAILPDGSLRILNASKSDEGKYVCRGENVFGSAEIIASLSVKEPTRIELTPKRTELTVGESIVLNCKAIHDASLDVTFYWTLKGQPIDFEEEGGHFESIRAQASSADLMIRNILLMHAGRYGCRVQTTADSVSDEAELLVRGPPGPPGIVIVEEITESTATLSWSPAADNHSPISSYNLQARSPFSLGWQTVKTVPEIITGDMESAMAVDLNPWVEYEFRVVATNPIGTGDPSTPSRMIRTNEAVPKTAPTNVSGRSGRRHELVIAWEPVSEEFQNGEGFGYIVAFRPNGTRGWKEKMVTSSEASKFIYRDESVPPLTPFEVKVGVYNNKGDGPFSQIVVICSAEGEPSAAPTDVKATSVSVSEILVAWKHIKESLGRPQGFEVGYWKDMEQEDTAETVKTRGNESFVILTGLEGNTLYHFTVRAYNGAGYGPPSSEVSATTKKSPPSQAPSNLRWEQQGSQVSLGWEPVIPLANESEVVGYKVFYRQEGHSNSQVIETQKLQAVVPLPDAGVYIIEVRAYSEGGDGTASSQIRVPSYSGGKITSAQSTLHSLSTSSSSVTLLLALMIPSTSW",
# BAIP3_HUMAN
"MRPRGAAFAAGPPGDLHLGTAIGFAGAIWRSRSPAMSTLLDIKSSVLRQVQVCPSFRRRTEQDPGSASADPQEPATGAWKPGDGVEFFAHMRLMLKKGEGRQGLPCLEVPLRSGSPAPPEPVDPSLGLRALAPEEVEMLYEEALYTVLYRAGTMGPDQVDDEEALLSYLQQVFGTSLEEHTEAIERVRKAKAPTYALKVSVMRAKNLLAKDPNGFSDPYCMLGILPASDATREPRAQKEQRFGFRKGSKRGGPLPAKCIQVTEVKSSTLNPVWKEHFLFEIEDVSTDQLHLDIWDHDDDVSLVEACRKLNEVIGLKGMGRYFKQIVKSARANGTAGPTEDHTDDFLGCLNIPVREVPVAGVDRWFKLEPRSSASRVQGHCHLVLKLITTQRDTAMSQRGRSGFLSHLLLLSHLLRLEHSAEEPNSSSWRGELSTPAATILCLHGAQSNLSPLQLAVLHWQVSSRHHQTCTLDYSYLLGLLEDMQAHWEEAPSLPQEQEESLADSLSAFSEFGLQLLRQLRDYFPATNSTAVHRLELLLKCLGKLQLFQPSFEICPFESELNMDIAAALKRGNREWYDRILNDKSPREQPGPQRLPGLVVLADAVYDDLQFCYSVYASLFHSILNVDVFTLTFRQLERLVAEEAWVLTEELSPKMTLEVASGLFELYLTLADLQRFWDSIPGRDSRSLALAGIHAPFLPAVKLWFQVLRDQAKWRLQGAVDMDTLEPVDASSRHSSSAATAGLCLSHIQELWVRLAWPDPAQAQGLGTQLGQDVCEATLFYTELLRKKVDTQPGAAGEAVSEALCVVLNNVELVRKAAGQALKGLAWPEGATGPEGVLPRPLLSCTQALDDDLQREAHTVTAHLTSKMVGDIRKYVQHISLSPDSIQNDEAVAPLMKYLDEKLALLNASLVKGNLSRVLEALWELLLQAILQALGANRDVSADFYSRFHFTLEALVSFFHAEGQGLPLESLRDGSYKRLKEELRLHKCSTRECIEQFYLDKLKQRTLEQNRFGRLSVRCHYEAAEQRLAVEVLHAADLLPLDANGLSDPFVIVELGPPHLFPLVRSQRTQVKTRTLHPVYDELFYFSVPAEACRRRAACVLFTVMDHDWLSTNDFAGEAALGLGGVTGVARPQVGGGARAGQPVTLHLCRPRAQVRSALRRLEGRTSKEAQEFVKKLKELEKCMEADP",
# SLIT2_HUMAN
"MRGVGWQMLSLSLGLVLAILNKVAPQACPAQCSCSGSTVDCHGLALRSVPRNIPRNTERLDLNGNNITRITKTDFAGLRHLRVLQLMENKISTIERGAFQDLKELERLRLNRNHLQLFPELLFLGTAKLYRLDLSENQIQAIPRKAFRGAVDIKNLQLDYNQISCIEDGAFRALRDLEVLTLNNNNITRLSVASFNHMPKLRTFRLHSNNLYCDCHLAWLSDWLRQRPRVGLYTQCMGPSHLRGHNVAEVQKREFVCSGHQSFMAPSCSVLHCPAACTCSNNIVDCRGKGLTEIPTNLPETITEIRLEQNTIKVIPPGAFSPYKKLRRIDLSNNQISELAPDAFQGLRSLNSLVLYGNKITELPKSLFEGLFSLQLLLLNANKINCLRVDAFQDLHNLNLLSLYDNKLQTIAKGTFSPLRAIQTMHLAQNPFICDCHLKWLADYLHTNPIETSGARCTSPRRLANKRIGQIKSKKFRCSAKEQYFIPGTEDYRSKLSGDCFADLACPEKCRCEGTTVDCSNQKLNKIPEHIPQYTAELRLNNNEFTVLEATGIFKKLPQLRKINFSNNKITDIEEGAFEGASGVNEILLTSNRLENVQHKMFKGLESLKTLMLRSNRITCVGNDSFIGLSSVRLLSLYDNQITTVAPGAFDTLHSLSTLNLLANPFNCNCYLAWLGEWLRKKRIVTGNPRCQKPYFLKEIPIQDVAIQDFTCDDGNDDNSCSPLSRCPTECTCLDTVVRCSNKGLKVLPKGIPRDVTELYLDGNQFTLVPKELSNYKHLTLIDLSNNRISTLSNQSFSNMTQLLTLILSYNRLRCIPPRTFDGLKSLRLLSLHGNDISVVPEGAFNDLSALSHLAIGANPLYCDCNMQWLSDWVKSEYKEPGIARCAGPGEMADKLLLTTPSKKFTCQGPVDVNILAKCNPCLSNPCKNDGTCNSDPVDFYRCTCPYGFKGQDCDVPIHACISNPCKHGGTCHLKEGEEDGFWCICADGFEGENCEVNVDDCEDNDCENNSTCVDGINNYTCLCPPEYTGELCEEKLDFCAQDLNPCQHDSKCILTPKGFKCDCTPGYVGEHCDIDFDDCQDNKCKNGAHCTDAVNGYTCICPEGYSGLFCEFSPPMVLPRTSPCDNFDCQNGAQCIVRINEPICQCLPGYQGEKCEKLVSVNFINKESYLQIPSAKVRPQTNITLQIATDEDSGILLYKGDKDHIAVELYRGRVRASYDTGSHPASAIYSVETINDGNFHIVELLALDQSLSLSVDGGNPKIITNLSKQSTLNFDSPLYVGGMPGKSNVASLRQAPGQNGTSFHGCIRNLYINSELQDFQKVPMQTGILPGCEPCHKKVCAHGTCQPSSQAGFTCECQEGWMGPLCDQRTNDPCLGNKCVHGTCLPINAFSYSCKCLEGHGGVLCDEEEDLFNPCQAIKCKHGKCRLSGLGQPYCECSSGYTGDSCDREISCRGERIRDYYQKQQGYAACQTTKKVSRLECRGGCAGGQCCGPLRSKRRKYSFECTDGSSFVDEVEKVVKCGCTRCVS",
# PKHG5_HUMAN
"MHYDGHVRFDLPPQGSVLARNVSTRSCPPRTSPAVDLEEEEEESSVDGKGDRKSTGLKLSKKKARRRHTDDPSKECFTLKFDLNVDIETEIVPAMKKKSLGEVLLPVFERKGIALGKVDIYLDQSNTPLSLTFEAYRFGGHYLRVKAPAKPGDEGKVEQGMKDSKSLSLPILRPAGTGPPALERVDAQSRRESLDILAPGRRRKNMSEFLGEASIPGQEPPTPSSCSLPSGSSGSTNTGDSWKNRAASRFSGFFSSGPSTSAFGREVDKMEQLEGKLHTYSLFGLPRLPRGLRFDHDSWEEEYDEDEDEDNACLRLEDSWRELIDGHEKLTRRQCHQQEAVWELLHTEASYIRKLRVIINLFLCCLLNLQESGLLCEVEAERLFSNIPEIAQLHRRLWASVMAPVLEKARRTRALLQPGDFLKGFKMFGSLFKPYIRYCMEEEGCMEYMRGLLRDNDLFRAYITWAEKHPQCQRLKLSDMLAKPHQRLTKYPLLLKSVLRKTEEPRAKEAVVAMIGSVERFIHHVNACMRQRQERQRLAAVVSRIDAYEVVESSSDEVDKLLKEFLHLDLTAPIPGASPEETRQLLLEGSLRMKEGKDSKMDVYCFLFTDLLLVTKAVKKAERTRVIRPPLLVDKIVCRELRDPGSFLLIYLNEFHSAVGAYTFQASGQALCRGWVDTIYNAQNQLQQLRAQEPPGSQQPLQSLEEEEDEQEEEEEEEEEEEEGEDSGTSAASSPTIMRKSSGSPDSQHCASDGSTETLAMVVVEPGDTLSSPEFDSGPFSSQSDETSLSTTASSATPTSELLPLGPVDGRSCSMDSAYGTLSPTSLQDFVAPGPMAELVPRAPESPRVPSPPPSPRLRRRTPVQLLSCPPHLLKSKSEASLLQLLAGAGTHGTPSAPSRSLSELCLAVPAPGIRTQGSPQEAGPSWDCRGAPSPGSGPGLVGCLAGEPAGSHRKRCGDLPSGASPRVQPEPPPGVSAQHRKLTLAQLYRIRTTLLLNSTLTASEV",
# DEND_HUMAN
"MLDGPLFSEGPDSPRELQDEESGSCLWVQKSKLLVIEVKTISCHYSRRAPSRQPMDFQASHWARGFQNRTCGPRPGSPQPPPRRPWASRVLQEATNWRAGPLAEVRAREQEKRKAASQEREAKETERKRRKAGGARRSPPGRPRPEPRNAPRVAQLAGLPAPLRPERLAPVGRAPRPSAQPQSDPGSAWAGPWGGRRPGPPSYEAHLLLRGSAGTAPRRRWDRPPPYVAPPSYEGPHRTLGTKRGPGNSQVPTSSAPAATPARTDGGRTKKRLDPRIYRDVLGAWGLRQGQGLLGGSPGCGAARARPEPGKGVVEKSLGLAAADLNSGSDSHPQAKATGSAGTEIAPAGSATAAPCAPHPAPRSRHHLKGSREGKEGEQIWFPKCWIPSPKKQPPRHSQTLPRPWAPGGTGWRESLGLGEGAGPETLEGWKATRRAHTLPRSSQGLSRGEGVFVIDATCVVIRSQYVPTPRTQQVQLLPSGVTRVVGDSPSQSKPGKEEGEGATVFPSPCQKRLSSSRLLHQPGGGRGGEAEGGRPGDSTLEERTFRILGLPAPEVNLRDAPTQPGSPEHQALGPAASGAQGRAEGSEVAVVQRRAGRGWARTPGPYAGALREAVSRIRRHTAPDSDTDEAEELSVHSGSSDGSDTEAPGASWRNERTLPEVGNSSPEEDGKTAELSDSVGEILDVISQTEEVLFGVRDIRGTQQGNRKRQ",
# AGRL1_HUMAN
"MARLAAVLWNLCVTAVLVTSATQGLSRAGLPFGLMRRELACEGYPIELRCPGSDVIMVENANYGRTDDKICDADPFQMENVQCYLPDAFKIMSQRCNNRTQCVVVAGSDAFPDPCPGTYKYLEVQYDCVPYKVEQKVFVCPGTLQKVLEPTSTHESEHQSGAWCKDPLQAGDRIYVMPWIPYRTDTLTEYASWEDYVAARHTTTYRLPNRVDGTGFVVYDGAVFYNKERTRNIVKYDLRTRIKSGETVINTANYHDTSPYRWGGKTDIDLAVDENGLWVIYATEGNNGRLVVSQLNPYTLRFEGTWETGYDKRSASNAFMVCGVLYVLRSVYVDDDSEAAGNRVDYAFNTNANREEPVSLTFPNPYQFISSVDYNPRDNQLYVWNNYFVVRYSLEFGPPDPSAGPATSPPLSTTTTARPTPLTSTASPAATTPLRRAPLTTHPVGAINQLGPDLPPATAPVPSTRRPPAPNLHVSPELFCEPREVRRVQWPATQQGMLVERPCPKGTRGIASFQCLPALGLWNPRGPDLSNCTSPWVNQVAQKIKSGENAANIASELARHTRGSIYAGDVSSSVKLMEQLLDILDAQLQALRPIERESAGKNYNKMHKRERTCKDYIKAVVETVDNLLRPEALESWKDMNATEQVHTATMLLDVLEEGAFLLADNVREPARFLAAKENVVLEVTVLNTEGQVQELVFPQEEYPRKNSIQLSAKTIKQNSRNGVVKVVFILYNNLGLFLSTENATVKLAGEAGPGGPGGASLVVNSQVIAASINKESSRVFLMDPVIFTVAHLEDKNHFNANCSFWNYSERSMLGYWSTQGCRLVESNKTHTTCACSHLTNFAVLMAHREIYQGRINELLLSVITWVGIVISLVCLAICISTFCFLRGLQTDRNTIHKNLCINLFLAELLFLVGIDKTQYEIACPIFAGLLHYFFLAAFSWLCLEGVHLYLLLVEVFESEYSRTKYYYLGGYCFPALVVGIAAAIDYRSYGTEKACWLRVDNYFIWSFIGPVSFVIVVNLVFLMVTLHKMIRSSSVLKPDSSRLDNIKSWALGAIALLFLLGLTWAFGLLFINKESVVMAYLFTTFNAFQGVFIFVFHCALQKKVHKEYSKCLRHSYCCIRSPPGGTHGSLKTSAMRSNTRYYTGTQSRIRRMWNDTVRKQTESSFMAGDINSTPTLNRGTMGNHLLTNPVLQPRGGTSPYNTLIAESVGFNPSSPPVFNSPGSYREPKHPLGGREACGMDTLPLNGNFNNSYSLRSGDFPPGDGGPEPPRGRNLADAAAFEKMIISELVHNNLRGSSSAAKGPPPPEPPVPPVPGGGGEEEAGGPGGADRAEIELLYKALEEPLLLPRAQSVLYQSDLDESESCTAEDGATSRPLSSPPGRDSLYASGANLRDSPSYPDSSPEGPSEALPPPPPAPPGPPEIYYTSRPPALVARNPLQGYYQVRRPSHEGYLAAPGLEGPGPDGDGQMQLVTSL",
# ABLM3_HUMAN
"MNTSIPYQQNPYNPRGSSNVIQCYRCGDTCKGEVVRVHNNHFHIRCFTCQVCGCGLAQSGFFFKNQEYICTQDYQQLYGTRCDSCRDFITGEVISALGRTYHPKCFVCSLCRKPFPIGDKVTFSGKECVCQTCSQSMASSKPIKIRGPSHCAGCKEEIKHGQSLLALDKQWHVSCFKCQTCSVILTGEYISKDGVPYCESDYHAQFGIKCETCDRYISGRVLEAGGKHYHPTCARCVRCHQMFTEGEEMYLTGSEVWHPICKQAARAEKKLKHRRTSETSISPPGSSIGSPNRVICAKVDNEILNYKDLAALPKVKSIYEVQRPDLISYEPHSRYMSDEMLERCGYGESLGTLSPYSQDIYENLDLRQRRASSPGYIDSPTYSRQGMSPTFSRSPHHYYRSGPESGRSSPYHSQLDVRSSTPTSYQAPKHFHIPAGDSNIYRKPPIYKRHGDLSTATKSKTSEDISQTSKYSPIYSPDPYYASESEYWTYHGSPKVPRARRFSSGGEEDDFDRSMHKLQSGIGRLILKEEMKARSSSYADPWTPPRSSTSSREALHTAGYEMSLNGSPRSHYLADSDPLISKSASLPAYRRNGLHRTPSADLFHYDSMNAVNWGMREYKIYPYELLLVTTRGRNRLPKDVDRTRLERHLSQEEFYQVFGMTISEFDRLALWKRNELKKQARLF",
# SLIK3_HUMAN
"MKPSIAEMLHRGRMLWIILLSTIALGWTTPIPLIEDSEEIDEPCFDPCYCEVKESLFHIHCDSKGFTNISQITEFWSRPFKLYLQRNSMRKLYTNSFLHLNNAVSINLGNNALQDIQTGAFNGLKILKRLYLHENKLDVFRNDTFLGLESLEYLQADYNVIKRIESGAFRNLSKLRVLILNDNLIPMLPTNLFKAVSLTHLDLRGNRLKVLFYRGMLDHIGRSLMELQLEENPWNCTCEIVQLKSWLERIPYTALVGDITCETPFHFHGKDLREIRKTELCPLLSDSEVEASLGIPHSSSSKENAWPTKPSSMLSSVHFTASSVEYKSSNKQPKPTKQPRTPRPPSTSQALYPGPNQPPIAPYQTRPPIPIICPTGCTCNLHINDLGLTVNCKERGFNNISELLPRPLNAKKLYLSSNLIQKIYRSDFWNFSSLDLLHLGNNRISYVQDGAFINLPNLKSLFLNGNDIEKLTPGMFRGLQSLHYLYFEFNVIREIQPAAFSLMPNLKLLFLNNNLLRTLPTDAFAGTSLARLNLRKNYFLYLPVAGVLEHLNAIVQIDLNENPWDCTCDLVPFKQWIETISSVSVVGDVLCRSPENLTHRDVRTIELEVLCPEMLHVAPAGESPAQPGDSHLIGAPTSASPYEFSPPGGPVPLSVLILSLLVLFFSAVFVAAGLFAYVLRRRRKKLPFRSKRQEGVDLTGIQMQCHRLFEDGGGGGGGSGGGGRPTLSSPEKAPPVGHVYEYIPHPVTQMCNNPIYKPREEEEVAVSSAQEAGSAERGGPGTQPPGMGEALLGSEQFAETPKENHSNYRTLLEKEKEWALAVSSSQLNTIVTVNHHHPHHPAVGGVSGVVGGTGGDLAGFRHHEKNGGVVLFPPGGGCGSGSMLLDRERPQPAPCTVGFVDCLYGTVPKLKELHVHPPGMQYPDLQQDARLKETLLFSAGKGFTDHQTQKSDYLELRAKLQTKPDYLEVLEKTTYRF",
# AP2A2_HUMAN
"MPAVSKGDGMRGLAVFISDIRNCKSKEAEIKRINKELANIRSKFKGDKALDGYSKKKYVCKLLFIFLLGHDIDFGHMEAVNLLSSNRYTEKQIGYLFISVLVNSNSELIRLINNAIKNDLASRNPTFMGLALHCIASVGSREMAEAFAGEIPKVLVAGDTMDSVKQSAALCLLRLYRTSPDLVPMGDWTSRVVHLLNDQHLGVVTAATSLITTLAQKNPEEFKTSVSLAVSRLSRIVTSASTDLQDYTYYFVPAPWLSVKLLRLLQCYPPPDPAVRGRLTECLETILNKAQEPPKSKKVQHSNAKNAVLFEAISLIIHHDSEPNLLVRACNQLGQFLQHRETNLRYLALESMCTLASSEFSHEAVKTHIETVINALKTERDVSVRQRAVDLLYAMCDRSNAPQIVAEMLSYLETADYSIREEIVLKVAILAEKYAVDYTWYVDTILNLIRIAGDYVSEEVWYRVIQIVINRDDVQGYAAKTVFEALQAPACHENLVKVGGYILGEFGNLIAGDPRSSPLIQFHLLHSKFHLCSVPTRALLLSTYIKFVNLFPEVKPTIQDVLRSDSQLRNADVELQQRAVEYLRLSTVASTDILATVLEEMPPFPERESSILAKLKKKKGPSTVTDLEDTKRDRSVDVNGGPEPAPASTSAVSTPSPSADLLGLGAAPPAPAGPPPSSGGSGLLVDVFSDSASVVAPLAPGSEDNFARFVCKNNGVLFENQLLQIGLKSEFRQNLGRMFIFYGNKTSTQFLNFTPTLICSDDLQPNLNLQTKPVDPTVEGGAQVQQVVNIECVSDFTEAPVLNIQFRYGGTFQNVSVQLPITLNKFFQPTEMASQDFFQRWKQLSNPQQEVQNIFKAKHPMDTEVTKAKIIGFGSALLEEVDPNPANFVGAGIIHTKTTQIGCLLRLEPNLQAQMYRLTLRTSKEAVSQRLCELLSAQF",
# CSTN1_HUMAN
"MLRRPAPALAPAARLLLAGLLCGGGVWAARVNKHKPWLEPTYHGIVTENDNTVLLDPPLIALDKDAPLRFAESFEVTVTKEGEICGFKIHGQNVPFDAVVVDKSTGEGVIRSKEKLDCELQKDYSFTIQAYDCGKGPDGTNVKKSHKATVHIQVNDVNEYAPVFKEKSYKATVIEGKQYDSILRVEAVDADCSPQFSQICSYEIITPDVPFTVDKDGYIKNTEKLNYGKEHQYKLTVTAYDCGKKRATEDVLVKISIKPTCTPGWQGWNNRIEYEPGTGALAVFPNIHLETCDEPVASVQATVELETSHIGKGCDRDTYSEKSLHRLCGAAAGTAELLPSPSGSLNWTMGLPTDNGHDSDQVFEFNGTQAVRIPDGVVSVSPKEPFTISVWMRHGPFGRKKETILCSSDKTDMNRHHYSLYVHGCRLIFLFRQDPSEEKKYRPAEFHWKLNQVCDEEWHHYVLNVEFPSVTLYVDGTSHEPFSVTEDYPLHPSKIETQLVVGACWQEFSGVENDNETEPVTVASAGGDLHMTQFFRGNLAGLTLRSGKLADKKVIDCLYTCKEGLDLQVLEDSGRGVQIQAHPSQLVLTLEGEDLGELDKAMQHISYLNSRQFPTPGIRRLKITSTIKCFNEATCISVPPVDGYVMVLQPEEPKISLSGVHHFARAASEFESSEGVFLFPELRIISTITREVEPEGDGAEDPTVQESLVSEEIVHDLDTCEVTVEGEELNHEQESLEVDMARLQQKGIEVSSSELGMTFTGVDTMASYEEVLHLLRYRNWHARSLLDRKFKLICSELNGRYISNEFKVEVNVIHTANPMEHANHMAAQPQFVHPEHRSFVDLSGHNLANPHPFAVVPSTATVVIVVCVSFLVFMIILGVFRIRAAHRRTMRDQDTGKENEMDWDDSALTITVNPMETYEDQHSSEEEEEEEEEEESEDGEEEDDITSAESESSEEEEGEQGDPQNATRQQQLEWDDSTLSY",
# ARHGF_HUMAN
"MSAQSLPAATPPTQKPPRIIRPRPPSRSRAAQSPGPPHNGSSPQELPRNSNDAPTPMCTPIFWEPPAASLKPPALLPPSASRASLDSQTSPDSPSSTPTPSPVSRRSASPEPAPRSPVPPPKPSGSPCTPLLPMAGVLAQNGSASAPGTVRRLAGRFEGGAEGRAQDADAPEPGLQARADVNGEREAPLTGSGSQENGAPDAGLACPPCCPCVCHTTRPGLELRWVPVGGYEEVPRVPRRASPLRTSRSRPHPPSIGHPAVVLTSYRSTAERKLLPLLKPPKPTRVRQDATIFGDPPQPDLDLLSEDGIQTGDSPDEAPQNTPPATVEGREEEGLEVLKEQNWELPLQDEPLYQTYRAAVLSEELWGVGEDGSPSPANAGDAPTFPRPPGPRNTLWQELPAVQASGLLDTLSPQERRMQESLFEVVTSEASYLRSLRLLTDTFVLSQALRDTLTPRDHHTLFSNVQRVQGVSERFLATLLSRVRSSPHISDLCDVVHAHAVGPFSVYVDYVRNQQYQEETYSRLMDTNVRFSAELRRLQSLPKCERLPLPSFLLLPFQRITRLRMLLQNILRQTEEGSSRQENAQKALGAVSKIIERCSAEVGRMKQTEELIRLTQRLRFHKVKALPLVSWSRRLEFQGELTELGCRRGGVLFASRPRFTPLCLLLFSDLLLITQPKSGQRLQVLDYAHRSLVQAQQVPDPSGPPTFRLSLLSNHQGRPTHRLLQASSLSDMQRWLGAFPTPGPLPCSPDTIYEDCDCSQELCSESSAPAKTEGRSLESRAAPKHLHKTPEGWLKGLPGAFPAQLVCEVTGEHERRRHLRQNQRLLEAVGSSSGTPNAPPP",
# SLIK5_HUMAN
"MHTCCPPVTLEQDLHRKMHSWMLQTLAFAVTSLVLSCAETIDYYGEICDNACPCEEKDGILTVSCENRGIISLSEISPPRFPIYHLLLSGNLLNRLYPNEFVNYTGASILHLGSNVIQDIETGAFHGLRGLRRLHLNNNKLELLRDDTFLGLENLEYLQVDYNYISVIEPNAFGKLHLLQVLILNDNLLSSLPNNLFRFVPLTHLDLRGNRLKLLPYVGLLQHMDKVVELQLEENPWNCSCELISLKDWLDSISYSALVGDVVCETPFRLHGRDLDEVSKQELCPRRLISDYEMRPQTPLSTTGYLHTTPASVNSVATSSSAVYKPPLKPPKGTRQPNKPRVRPTSRQPSKDLGYSNYGPSIAYQTKSPVPLECPTACSCNLQISDLGLNVNCQERKIESIAELQPKPYNPKKMYLTENYIAVVRRTDFLEATGLDLLHLGNNRISMIQDRAFGDLTNLRRLYLNGNRIERLSPELFYGLQSLQYLFLQYNLIREIQSGTFDPVPNLQLLFLNNNLLQAMPSGVFSGLTLLRLNLRSNHFTSLPVSGVLDQLKSLIQIDLHDNPWDCTCDIVGMKLWVEQLKVGVLVDEVICKAPKKFAETDMRSIKSELLCPDYSDVVVSTPTPSSIQVPARTSAVTPAVRLNSTGAPASLGAGGGASSVPLSVLILSLLLVFIMSVFVAAGLFVLVMKRRKKNQSDHTSTNNSDVSSFNMQYSVYGGGGGTGGHPHAHVHHRGPALPKVKTPAGHVYEYIPHPLGHMCKNPIYRSREGNSVEDYKDLHELKVTYSSNHHLQQQQQPPPPPQQPQQQPPPQLQLQPGEEERRESHHLRSPAYSVSTIEPREDLLSPVQDADRFYRGILEPDKHCSTTPAGNSLPEYPKFPCSPAAYTFSPNYDLRRPHQYLHPGAGDSRLREPVLYSPPSAVFVEPNRNEYLELKAKLNVEPDYLEVLEKQTTFSQF",
# KCNK2_HUMAN
"MLPSASRERPGYRAGVAAPDLLDPKSAAQNSKPRLSFSTKPTVLASRVESDTTINVMKWKTVSTIFLVVVLYLIIGATVFKALEQPHEISQRTTIVIQKQTFISQHSCVNSTELDELIQQIVAAINAGIIPLGNTSNQISHWDLGSSFFFAGTVITTIGFGNISPRTEGGKIFCIIYALLGIPLFGFLLAGVGDQLGTIFGKGIAKVEDTFIKWNVSQTKIRIISTIIFILFGCVLFVALPAIIFKHIEGWSALDAIYFVVITLTTIGFGDYVAGGSDIEYLDFYKPVVWFWILVGLAYFAAVLSMIGDWLRVISKKTKEEVGEFRAHAAEWTANVTAEFKETRRRLSVEIYDKFQRATSIKRKLSAELAGNHNQELTPCRRTLSVNHLTSERDVLPPLLKTESIYLNGLTPHCAGEEIAVIENIK",
# S1PR2_HUMAN
"MGSLYSEYLNPNKVQEHYNYTKETLETQETTSRQVASAFIVILCCAIVVENLLVLIAVARNSKFHSAMYLFLGNLAASDLLAGVAFVANTLLSGSVTLRLTPVQWFAREGSAFITLSASVFSLLAIAIERHVAIAKVKLYGSDKSCRMLLLIGASWLISLVLGGLPILGWNCLGHLEACSTVLPLYAKHYVLCVVTIFSIILLAIVALYVRIYCVVRSSHADMAAPQTLALLKTVTIVLGVFIVCWLPAFSILLLDYACPVHSCPILYKAHYFFAVSTLNSLLNPVIYTWRSRDLRREVLRPLQCWRPGVGVQGRRRGGTPGHHLLPLRSSSSLERGMHMPTSPTFLEGNTVV",
# RIMB1_HUMAN
"MEQLTTLPRPGDPGAMEPWALPTWHSWTPGRGGEPSSAAPSIADTPPAALQLQELRSEESSKPKGDGSSRPVGGTDPEGAEACLPSLGQQASSSGPACQRPEDEEVEAFLKAKLNMSFGDRPNLELLRALGELRQRCAILKEENQMLRKSSFPETEEKVRRLKRKNAELAVIAKRLEERARKLQETNLRVVSAPLPRPGTSLELCRKALARQRARDLSETASALLAKDKQIAALQRECRELQARLTLVGKEGPQWLHVRDFDRLLRESQREVLRLQRQIALRNQRETLPLPPSWPPGPALQARAGAPAPGAPGEATPQEDADNLPVILGEPEKEQRVQQLESELSKKRKKCESLEQEARKKQRRCEELELQLRQAQNENARLVEENSRLSGRATEKEQVEWENAELRGQLLGVTQERDSALRKSQGLQSKLESLEQVLKHMREVAQRRQQLEVEHEQARLSLREKQEEVRRLQQAQAEAQREHEGAVQLLESTLDSMQARVRELEEQCRSQTEQFSLLAQELQAFRLHPGPLDLLTSALDCGSLGDCPPPPCCCSIPQPCRGSGPKDLDLPPGSPGRCTPKSSEPAPATLTGVPRRTAKKAESLSNSSHSESIHNSPKSCPTPEVDTASEVEELEADSVSLLPAAPEGSRGGARIQVFLARYSYNPFEGPNENPEAELPLTAGEYIYIYGNMDEDGFFEGELMDGRRGLVPSNFVERVSDDDLLTSLPPELADLSHSSGPELSFLSVGGGGSSSGGQSSVGRSQPRPEEEDAGDELSLSPSPEGLGEPPAVPYPRRLVVLKQLAHSVVLAWEPPPEQVELHGFHICVNGELRQALGPGAPPKAVLENLDLWAGPLHISVQALTSRGSSDPLRCCLAVGARAGVVPSQLRVHRLTATSAEITWVPGNSNLAHAIYLNGEECPPASPSTYWATFCHLRPGTPYQAQVEAQLPPQGPWEPGWERLEQRAATLQFTTLPAGPPDAPLDVQIEPGPSPGILIISWLPVTIDAAGTSNGVRVTGYAIYADGQKIMEVASPTAGSVLVELSQLQLLQVCREVVVRTMSPHGESADSIPAPITPALAPASLPARVSCPSPHPSPEARAPLASASPGPGDPSSPLQHPAPLGTQEPPGAPPASPSREMAKGSHEDPPAPCSQEEAGAAVLGTSEERTASTSTLGEKDPGPAAPSLAKQEAEWTAGEACPASSSTQGARAQQAPNTEMCQGGDPGSGLRPRAEKEDTAELGVHLVNSLVDHGRNSDLSDIQEEEEEEEEEEEEELGSRTCSFQKQVAGNSIRENGAKSQPDPFCETDSDEEILEQILELPLQQFCSKKLFSIPEEEEEEEEDEEEEKSGAGCSSRDPGPPEPALLGLGCDSGQPRRPGQCPLSPESSRAGDCLEDMPGLVGGSSRRRGGGSPEKPPSRRRPPDPREHCSRLLSNNGPQASGRLGPTRERGGLPVIEGPRTGLEASGRGRLGPSRRCSRGRALEPGLASCLSPKCLEISIEYDSEDEQEAGSGGISITSSCYPGDGEAWGTATVGRPRGPPKANSGPKPYPRLPAWEKGEPERRGRSATGRAKEPLSRATETGEARGQDGSGRRGPQKRGVRVLRPSTAELVPARSPSETLAYQHLPVRIFVALFDYDPVSMSPNPDAGEEELPFREGQILKVFGDKDADGFYQGEGGGRTGYIPCNMVAEVAVDSPAGRQQLLQRGYLSPDILLEGSGNGPFVYSTAHTTGPPPKPRRSKKAESEGPAQPCPGPPKLVPSADLKAPHSMVAAFDYNPQESSPNMDVEAELPFRAGDVITVFGGMDDDGFYYGELNGQRGLVPSNFLEGPGPEAGGLDREPRTPQAESQRTRRRRVQC",
# NXPH4_HUMAN
"MRLLPEWFLLLFGPWLLRKAVSAQIPESGRPQYLGLRPAAAGAGAPGQQLPEPRSSDGLGVGRAWSWAWPTNHTGALARAGAAGALPAQRTKRKPSIKAARAKKIFGWGDFYFRVHTLKFSLLVTGKIVDHVNGTFSVYFRHNSSSLGNLSVSIVPPSKRVEFGGVWLPGPVPHPLQSTLALEGVLPGLGPPLGMAAAAAGPGLGGSLGGALAGPLGGALGVPGAKESRAFNCHVEYEKTNRARKHRPCLYDPSQVCFTEHTQSQAAWLCAKPFKVICIFVSFLSFDYKLVQKVCPDYNFQSEHPYFG",
# GBRAP_HUMAN
"MKFVYKEEHPFEKRRSEGEKIRKKYPDRVPVIVEKAPKARIGDLDKKKYLVPSDLTVGQFYFLIRKRIHLRAEDALFFFVNNVIPPTSATMGQLYQEHHEEDFFLYIAYSDESVYGL",
# CAC1H_HUMAN
"MTEGARAADEVRVPLGAPPPGPAALVGASPESPGAPGREAERGSELGVSPSESPAAERGAELGADEEQRVPYPALAATVFFCLGQTTRPRSWCLRLVCNPWFEHVSMLVIMLNCVTLGMFRPCEDVECGSERCNILEAFDAFIFAFFAVEMVIKMVALGLFGQKCYLGDTWNRLDFFIVVAGMMEYSLDGHNVSLSAIRTVRVLRPLRAINRVPSMRILVTLLLDTLPMLGNVLLLCFFVFFIFGIVGVQLWAGLLRNRCFLDSAFVRNNNLTFLRPYYQTEEGEENPFICSSRRDNGMQKCSHIPGRRELRMPCTLGWEAYTQPQAEGVGAARNACINWNQYYNVCRSGDSNPHNGAINFDNIGYAWIAIFQVITLEGWVDIMYYVMDAHSFYNFIYFILLIIVGSFFMINLCLVVIATQFSETKQRESQLMREQRARHLSNDSTLASFSEPGSCYEELLKYVGHIFRKVKRRSLRLYARWQSRWRKKVDPSAVQGQGPGHRQRRAGRHTASVHHLVYHHHHHHHHHYHFSHGSPRRPGPEPGACDTRLVRAGAPPSPPSPGRGPPDAESVHSIYHADCHIEGPQERARVAHAAATAAASLRLATGLGTMNYPTILPSGVGSGKGSTSPGPKGKWAGGPPGTGGHGPLSLNSPDPYEKIPHVVGEHGLGQAPGHLSGLSVPCPLPSPPAGTLTCELKSCPYCTRALEDPEGELSGSESGDSDGRGVYEFTQDVRHGDRWDPTRPPRATDTPGPGPGSPQRRAQQRAAPGEPGWMGRLWVTFSGKLRRIVDSKYFSRGIMMAILVNTLSMGVEYHEQPEELTNALEISNIVFTSMFALEMLLKLLACGPLGYIRNPYNIFDGIIVVISVWEIVGQADGGLSVLRTFRLLRVLKLVRFLPALRRQLVVLVKTMDNVATFCTLLMLFIFIFSILGMHLFGCKFSLKTDTGDTVPDRKNFDSLLWAIVTVFQILTQEDWNVVLYNGMASTSSWAALYFVALMTFGNYVLFNLLVAILVEGFQAEGDANRSDTDEDKTSVHFEEDFHKLRELQTTELKMCSLAVTPNGHLEGRGSLSPPLIMCTAATPMPTPKSSPFLDAAPSLPDSRRGSSSSGDPPLGDQKPPASLRSSPCAPWGPSGAWSSRRSSWSSLGRAPSLKRRGQCGERESLLSGEGKGSTDDEAEDGRAAPGPRATPLRRAESLDPRPLRPAALPPTKCRDRDGQVVALPSDFFLRIDSHREDAAELDDDSEDSCCLRLHKVLEPYKPQWCRSREAWALYLFSPQNRFRVSCQKVITHKMFDHVVLVFIFLNCVTIALERPDIDPGSTERVFLSVSNYIFTAIFVAEMMVKVVALGLLSGEHAYLQSSWNLLDGLLVLVSLVDIVVAMASAGGAKILGVLRVLRLLRTLRPLRVISRAPGLKLVVETLISSLRPIGNIVLICCAFFIIFGILGVQLFKGKFYYCEGPDTRNISTKAQCRAAHYRWVRRKYNFDNLGQALMSLFVLSSKDGWVNIMYDGLDAVGVDQQPVQNHNPWMLLYFISFLLIVSFFVLNMFVGVVVENFHKCRQHQEAEEARRREEKRLRRLERRRRSTFPSPEAQRRPYYADYSPTRRSIHSLCTSHYLDLFITFIICVNVITMSMEHYNQPKSLDEALKYCNYVFTIVFVFEAALKLVAFGFRRFFKDRWNQLDLAIVLLSLMGITLEEIEMSAALPINPTIIRIMRVLRIARVLKLLKMATGMRALLDTVVQALPQVGNLGLLFMLLFFIYAALGVELFGRLECSEDNPCEGLSRHATFSNFGMAFLTLFRVSTGDNWNGIMKDTLRECSREDKHCLSYLPALSPVYFVTFVLVAQFVLVNVVVAVLMKHLEESNKEAREDAELDAEIELEMAQGPGSARRVDADRPPLPQESPGARDAPNLVARKVSVSRMLSLPNDSYMFRPVVPASAPHPRPLQEVEMETYGAGTPLGSVASVHSPPAESCASLQIPLAVSSPARSGEPLHALSPRGTARSPSLSRLLCRQEAVHTDSLEGKIDSPRDTLDPAEPGEKTPVRPVTQGGSLQSPPRSPRPASVRTRKHTFGQRCVSSRPAAPGGEEAEASDPADEEVSHITSSACPWQPTAEPHGPEASPVAGGERDLRRLYSVDAQGFLDKPGRADEQWRPSAELGSGEPGEAKAWGPEAEPALGARRKKKMSPPCISVEPPAEDEGSARPSAAEGGSTTLRRRTPSCEATPHRDSLEPTEGSGAGGDPAAKGERWGQASCRAEHLTVPSFAFEPLDLGVPSGDPFLDGSHSVTPESRASSSGAIVPLEPPESEPPMPVGDPPEKRRGLYLTVPQCPLEKPGSPSATPAPGGGADDPV",
# CSPG5_HUMAN
"MGRAGGGGPGRGPPPLLLFLGAALVLASGAVPAREAGSAVEAEELVKGSPAWEPPANDTREEAGPPAAGEDEASWTAPGGELAGPEEVLQESAAVTGTAWLEADSPGLGGVTAEAGSGDAQALPATLQAPHEVLGQSIMPPAIPEATEASGPPSPTPGDKLSPASELPKESPLEVWLNLGGSTPDPQGPELTYPFQGTLEPQPASDIIDIDYFEGLDGEGRGADLGSFPGSPGTSENHPDTEGETPSWSLLDLYDDFTPFDESDFYPTTSFYDDLDEEEEEEEDDKDAVGGGDLEDENELLVPTGKPGLGPGTGQPTSRWHAVPPQHTLGSVPGSSIALRPRPGEPGRDLASSENGTECRSGFVRHNGSCRSVCDLFPSYCHNGGQCYLVENIGAFCRCNTQDYIWHKGMRCESIITDFQVMCVAVGSAALVLLLLFMMTVFFAKKLYLLKTENTKLRRTNKFRTPSELHNDNFSLSTIAEGSHPNVRKLCNTPRTSSPHARALAHYDNVICQDDPSAPHKIQEVLKSCLKEEESFNIQNSMSPKLEGGKGDQADLDVNCLQNNLT",
# RTN3_HUMAN
"MAEPSAATQSHSISSSSFGAEPSAPGGGGSPGACPALGTKSCSSSCADSFVSSSSSQPVSLFSTSQEGLSSLCSDEPSSEIMTSSFLSSSEIHNTGLTILHGEKSHVLGSQPILAKEGKDHLDLLDMKKMEKPQGTSNNVSDSSVSLAAGVHCDRPSIPASFPEHPAFLSKKIGQVEEQIDKETKNPNGVSSREAKTALDADDRFTLLTAQKPPTEYSKVEGIYTYSLSPSKVSGDDVIEKDSPESPFEVIIDKAAFDKEFKDSYKESTDDFGSWSVHTDKESSEDISETNDKLFPLRNKEAGRYPMSALLSRQFSHTNAALEEVSRCVNDMHNFTNEILTWDLVPQVKQQTDKSSDCITKTTGLDMSEYNSEIPVVNLKTSTHQKTPVCSIDGSTPITKSTGDWAEASLQQENAITGKPVPDSLNSTKEFSIKGVQGNMQKQDDTLAELPGSPPEKCDSLGSGVATVKVVLPDDHLKDEMDWQSSALGEITEADSSGESDDTVIEDITADTSFENNKIQAEKPVSIPSAVVKTGEREIKEIPSCEREEKTSKNFEELVSDSELHQDQPDILGRSPASEAACSKVPDTNVSLEDVSEVAPEKPITTENPKLPSTVSPNVFNETEFSLNVTTSAYLESLHGKNVKHIDDSSPEDLIAAFTETRDKGIVDSERNAFKAISEKMTDFKTTPPVEVLHENESGGSEIKDIGSKYSEQSKETNGSEPLGVFPTQGTPVASLDLEQEQLTIKALKELGERQVEKSTSAQRDAELPSEEVLKQTFTFAPESWPQRSYDILERNVKNGSDLGISQKPITIRETTRVDAVSSLSKTELVKKHVLARLLTDFSVHDLIFWRDVKKTGFVFGTTLIMLLSLAAFSVISVVSYLILALLSVTISFRIYKSVIQAVQKSEEGHPFKAYLDVDITLSSEAFHNYMNAAMVHINRALKLIIRLFLVEDLVDSLKLAVFMWLMTYVGAVFNGITLLILAELLIFSVPIVYEKYKTQIDHYVGIARDQTKSIVEKIQAKLPGIAKKKAE",
# PCDH8_HUMAN
"MSPVRRWGSPCLFPLQLFSLCWVLSVAQSKTVRYSTFEEDAPGTVIGTLAEDLHMKVSGDTSFRLMKQFNSSLLRVREGDGQLTVGDAGLDRERLCGQAPQCVLAFDVVSFSQEQFRLVHVEVEVRDVNDHAPRFPRAQIPVEVSEGAAVGTRIPLEVPVDEDVGANGLQTVRLAEPHSPFRVELQTRADGAQCADLVLLQELDRESQAAYSLELVAQDGGRPPRSATAALSVRVLDANDHSPAFPQGAVAEVELAEDAPVGSLLLDLDAADPDEGPNGDVVFAFGARTPPEARRLFRLDPRSGRLTLAGPVDYERQDTYELDVRAQDRGPGPRAATCKVIVRIRDVNDNAPDIAITPLAAPGAPATSPFAAAAAAAALGGADASSPAGAGTPEAGATSLVPEGAARESLVALVSTSDRDSGANGQVRCALYGHEHFRLQPAYAGSYLVVTAASLDRERIAEYNLTLVAEDRGAPPLRTVRPYTVRVGDENDNAPLFTRPVYEVSVRENNPPGAYLATVAARDRDLGRNGQVTYRLLEAEVGRAGGAVSTYVSVDPATGAIYALRSFDYETLRQLDVRIQASDGGSPQLSSSALVQVRVLDQNDHAPVLVHPAPANGSLEVAVPGRTAKDTVVARVQARDADEGANGELAFELQQQEPREAFAIGRRTGEILLTGDLSQEPPGRVFRALLVISDGGRPPLTTTATVSFVVTAGGGRGPAAPASAGSPERSRPPGSRLGVSGSVLQWDTPLIVIIVLAGSCTLLLAAIIAIATTCNRRKKEVRKGGALREERPGAAGGGASAPGSPEEAARGAGPRPNMFDVLTFPGTGKAPFGSPAADAPPPAVAAAEVPGSEGGSATGESACHFEGQQRLRGAHAEPYGASPGFGKEPAPPVAVWKGHSFNTISGREAEKFSGKDSGKGDSDFNDSDSDISGDALKKDLINHMQSGLWACTAECKILGHSDRCWSPSCSGPNAHPSPHPPAQMSTFCKSTSLPRDPLRRDNYYQAQLPKTVGLQSVYEKVLHRDYDRTVTLLSPPRPGRLPDLQEIGVPLYQSPPGRYLSPKKGANENV",
# EPN2_HUMAN
"MTTSSIRRQMKNIVNNYSEAEIKVREATSNDPWGPSSSLMTEIADLTYNVVAFSEIMSMVWKRLNDHGKNWRHVYKALTLLDYLIKTGSERVAQQCRENIFAIQTLKDFQYIDRDGKDQGINVREKSKQLVALLKDEERLKAERAQALKTKERMAQVATGMGSNQITFGRGSSQPNLSTSHSEQEYGKAGGSPASYHGSPEASLCPQHRTGAPLGQSEELQPLSQRHPFLPHLGLASRPNGDWSQPCLTCDRAARATSPRVSSELEQARPQTSGEEELQLQLALAMSREVAEQEERLRRGDDLRLQMALEESRRDTVKIPKKKEHGSLPQQTTLLDLMDALPSSGPAAQKAEPWGPSASTNQTNPWGGPAAPASTSDPWPSFGTKPAASIDPWGVPTGATVQSVPKNSDPWAASQQPASSAGKRASDAWGAVSTTKPVSVSGSFELFSNLNGTIKDDFSEFDNLRTSKKTAESVTSLPSQNNGTTSPDPFESQPLTVASSKPSSARKTPESFLGPNAALVNLDSLVTRPAPPAQSLNPFLAPGAPATSAPVNPFQVNQPQPLTLNQLRGSPVLGTSTSFGPGPGVESMAVASMTSAAPQPALGATGSSLTPLGPAMMNMVGSVGIPPSAAQATGTTNPFLL",
# SNX4_HUMAN
"MEQAPPDPERQLQPAPLEPLGSPDAGLGAAVGKEAEGAGEESSGVDTMTHNNFWLKKIEISVSEAEKRTGRNAMNMQETYTAYLIETRSVEHTDGQSVLTDSLWRRYSEFELLRSYLLVYYPHIVVPPLPEKRAEFVWHKLSADNMDPDFVERRRIGLENFLLRIASHPILCRDKIFYLFLTQEGNWKETVNETGFQLKADSRLKALNATFRVKNPDKRFTDLKHYSDELQSVISHLLRVRARVADRLYGVYKVHGNYGRVFSEWSAIEKEMGDGLQSAGHHMDVYASSIDDILEDEEHYADQLKEYLFYAEALRAVCRKHELMQYDLEMAAQDLASKKQQCEELVTGTVRTFSLKGMTTKLFGQETPEQREARIKVLEEQINEGEQQLKSKNLEGREFVKNAWADIERFKEQKNRDLKEALISYAVMQISMCKKGIQVWTNAKECFSKM",
# KCNH1_HUMAN
"MTMAGGRRGLVAPQNTFLENIVRRSNDTNFVLGNAQIVDWPIVYSNDGFCKLSGYHRAEVMQKSSTCSFMYGELTDKDTIEKVRQTFENYEMNSFEILMYKKNRTPVWFFVKIAPIRNEQDKVVLFLCTFSDITAFKQPIEDDSCKGWGKFARLTRALTSSRGVLQQLAPSVQKGENVHKHSRLAEVLQLGSDILPQYKQEAPKTPPHIILHYCVFKTTWDWIILILTFYTAILVPYNVSFKTRQNNVAWLVVDSIVDVIFLVDIVLNFHTTFVGPAGEVISDPKLIRMNYLKTWFVIDLLSCLPYDVINAFENVDEVSAFMGDPGKIGFADQIPPPLEGRESQGISSLFSSLKVVRLLRLGRVARKLDHYIEYGAAVLVLLVCVFGLAAHWMACIWYSIGDYEIFDEDTKTIRNNSWLYQLAMDIGTPYQFNGSGSGKWEGGPSKNSVYISSLYFTMTSLTSVGFGNIAPSTDIEKIFAVAIMMIGSLLYATIFGNVTTIFQQMYANTNRYHEMLNSVRDFLKLYQVPKGLSERVMDYIVSTWSMSRGIDTEKVLQICPKDMRADICVHLNRKVFKEHPAFRLASDGCLRALAMEFQTVHCAPGDLIYHAGESVDSLCFVVSGSLEVIQDDEVVAILGKGDVFGDVFWKEATLAQSCANVRALTYCDLHVIKRDALQKVLEFYTAFSHSFSRNLILTYNLRKRIVFRKISDVKREEEERMKRKNEAPLILPPDHPVRRLFQRFRQQKEARLAAERGGRDLDDLDVEKGNVLTEHASANHSLVKASVVTVRESPATPVSFQAASTSGVPDHAKLQAPGSECLGPKGGGGDCAKRKSWARFKDACGKSEDWNKVSKAESMETLPERTKASGEATLKKTDSCDSGITKSDLRLDNVGEARSPQDRSPILAEVKHSFYPIPEQTLQATVLEVRHELKEDIKALNAKMTNIEKQLSEILRILTSRRSSQSPQELFEISRPQSPESERDIFGAS",
# VAPB_HUMAN
"MAKVEQVLSLEPQHELKFRGPFTDVVTTNLKLGNPTDRNVCFKVKTTAPRRYCVRPNSGIIDAGASINVSVMLQPFDYDPNEKSKHKFMVQSMFAPTDTSDMEAVWKEAKPEDLMDSKLRCVFELPAENDKPHDVEINKIISTTASKTETPIVSKSLSSSLDDTEVKKVMEECKRLQGEVQRLREENKQFKEEDGLRMRKTVQSNSPISALAPTGKEEGLSTRLLALVVLFFIVGVIIGKIAL",
# SNAPN_HUMAN
"MAGAGSAAVSGAGTPVAGPTGRDLFAEGLLEFLRPAVQQLDSHVHAVRESQVELREQIDNLATELCRINEDQKVALDLDPYVKKLLNARRRVVLVNNILQNAQERLRRLNHSVAKETARRRAMLDSGIYPPGSPGK",
# ATG7_HUMAN
"MAAATGDPGLSKLQFAPFSSALDVGFWHELTQKKLNEYRLDEAPKDIKGYYYNGDSAGLPARLTLEFSAFDMSAPTPARCCPAIGTLYNTNTLESFKTADKKLLLEQAANEIWESIKSGTALENPVLLNKFLLLTFADLKKYHFYYWFCYPALCLPESLPLIQGPVGLDQRFSLKQIEALECAYDNLCQTEGVTALPYFLIKYDENMVLVSLLKHYSDFFQGQRTKITIGVYDPCNLAQYPGWPLRNFLVLAAHRWSSSFQSVEVVCFRDRTMQGARDVAHSIIFEVKLPEMAFSPDCPKAVGWEKNQKGGMGPRMVNLSECMDPKRLAESSVDLNLKLMCWRLVPTLDLDKVVSVKCLLLGAGTLGCNVARTLMGWGVRHITFVDNAKISYSNPVRQPLYEFEDCLGGGKPKALAAADRLQKIFPGVNARGFNMSIPMPGHPVNFSSVTLEQARRDVEQLEQLIESHDVVFLLMDTRESRWLPAVIAASKRKLVINAALGFDTFVVMRHGLKKPKQQGAGDLCPNHPVASADLLGSSLFANIPGYKLGCYFCNDVVAPGDSTRDRTLDQQCTVSRPGLAVIAGALAVELMVSVLQHPEGGYAIASSSDDRMNEPPTSLGLVPHQIRGFLSRFDNVLPVSLAFDKCTACSSKVLDQYEREGFNFLAKVFNSSHSFLEDLTGLTLLHQETQAAEIWDMSDDETI",
# PARN_HUMAN
"MEIIRSNFKSNLHKVYQAIEEADFFAIDGEFSGISDGPSVSALTNGFDTPEERYQKLKKHSMDFLLFQFGLCTFKYDYTDSKYITKSFNFYVFPKPFNRSSPDVKFVCQSSSIDFLASQGFDFNKVFRNGIPYLNQEEERQLREQYDEKRSQANGAGALSYVSPNTSKCPVTIPEDQKKFIDQVVEKIEDLLQSEENKNLDLEPCTGFQRKLIYQTLSWKYPKGIHVETLETEKKERYIVISKVDEEERKRREQQKHAKEQEELNDAVGFSRVIHAIANSGKLVIGHNMLLDVMHTVHQFYCPLPADLSEFKEMTTCVFPRLLDTKLMASTQPFKDIINNTSLAELEKRLKETPFNPPKVESAEGFPSYDTASEQLHEAGYDAYITGLCFISMANYLGSFLSPPKIHVSARSKLIEPFFNKLFLMRVMDIPYLNLEGPDLQPKRDHVLHVTFPKEWKTSDLYQLFSAFGNIQISWIDDTSAFVSLSQPEQVKIAVNTSKYAESYRIQTYAEYMGRKQEEKQIKRKWTEDSWKEADSKRLNPQCIPYTLQNHYYRNNSFTAPSTVGKRNLSPSQEEAGLEDGVSGEISDTELEQTDSCAEPLSEGRKKAKKLKRMKKELSPAGSISKNSPATLFEVPDTW",
# GNAS3_HUMAN
"MDRRSRAQQWRRARHNYNDLCPPIGRRAATALLWLSCSIALLRALATSNARAQQRAAAQQRRSFLNAHHRSGAQVFPESPESESDHEHEEADLELSLPECLEYEEEFDYETESETESEIESETDFETEPETAPTTEPETEPEDDRGPVVPKHSTFGQSLTQRLHALKLRSPDASPSRAPPSTQEPQSPREGEELKPEDKDPRDPEESKEPKEEKQRRRCKPKKPTRRDASPESPSKKGPIPIRRH",
# AGRL2_HUMAN
"MVSSGCRMRSLWFIIVISFLPNTEGFSRAALPFGLVRRELSCEGYSIDLRCPGSDVIMIESANYGRTDDKICDADPFQMENTDCYLPDAFKIMTQRCNNRTQCIVVTGSDVFPDPCPGTYKYLEVQYECVPYIFVCPGTLKAIVDSPCIYEAEQKAGAWCKDPLQAADKIYFMPWTPYRTDTLIEYASLEDFQNSRQTTTYKLPNRVDGTGFVVYDGAVFFNKERTRNIVKFDLRTRIKSGEAIINYANYHDTSPYRWGGKTDIDLAVDENGLWVIYATEQNNGMIVISQLNPYTLRFEATWETVYDKRAASNAFMICGVLYVVRSVYQDNESETGKNSIDYIYNTRLNRGEYVDVPFPNQYQYIAAVDYNPRDNQLYVWNNNFILRYSLEFGPPDPAQVPTTAVTITSSAELFKTIISTTSTTSQKGPMSTTVAGSQEGSKGTKPPPAVSTTKIPPITNIFPLPERFCEALDSKGIKWPQTQRGMMVERPCPKGTRGTASYLCMISTGTWNPKGPDLSNCTSHWVNQLAQKIRSGENAASLANELAKHTKGPVFAGDVSSSVRLMEQLVDILDAQLQELKPSEKDSAGRSYNKLQKREKTCRAYLKAIVDTVDNLLRPEALESWKHMNSSEQAHTATMLLDTLEEGAFVLADNLLEPTRVSMPTENIVLEVAVLSTEGQIQDFKFPLGIKGAGSSIQLSANTVKQNSRNGLAKLVFIIYRSLGQFLSTENATIKLGADFIGRNSTIAVNSHVISVSINKESSRVYLTDPVLFTLPHIDPDNYFNANCSFWNYSERTMMGYWSTQGCKLVDTNKTRTTCACSHLTNFAILMAHREIAYKDGVHELLLTVITWVGIVISLVCLAICIFTFCFFRGLQSDRNTIHKNLCINLFIAEFIFLIGIDKTKYAIACPIFAGLLHFFFLAAFAWMCLEGVQLYLMLVEVFESEYSRKKYYYVAGYLFPATVVGVSAAIDYKSYGTEKACWLHVDNYFIWSFIGPVTFIILLNIIFLVITLCKMVKHSNTLKPDSSRLENIKSWVLGAFALLCLLGLTWSFGLLFINEETIVMAYLFTIFNAFQGVFIFIFHCALQKKVRKEYGKCFRHSYCCGGLPTESPHSSVKASTTRTSARYSSGTQSRIRRMWNDTVRKQSESSFISGDINSTSTLNQGMTGNYLLTNPLLRPHGTNNPYNTLLAETVVCNAPSAPVFNSPGHSLNNARDTSAMDTLPLNGNFNNSYSLHKGDYNDSVQVVDCGLSLNDTAFEKMIISELVHNNLRGSSKTHNLELTLPVKPVIGGSSSEDDAIVADASSLMHSDNPGLELHHKELEAPLIPQRTHSLLYQPQKKVKSEGTDSYVSQLTAEAEDHLQSPNRDSLYTSMPNLRDSPYPESSPDMEEDLSPSRRSENEDIYYKSMPNLGAGHQLQMCYQISRGNSDGYIIPINKEGCIPEGDVREGQMQLVTSL",
# NPTXR_HUMAN
"MKFLAVLLAAGMLAFLGAVICIIASVPLAASPARALPGGADNASVASGAAASPGPQRSLSALHGAGGSAGPPALPGAPAASAHPLPPGPLFSRFLCTPLAAACPSGAQQGDAAGAAPGEREELLLLQSTAEQLRQTALQQEARIRADQDTIRELTGKLGRCESGLPRGLQGAGPRRDTMADGPWDSPALILELEDAVRALRDRIDRLEQELPARVNLSAAPAPVSAVPTGLHSKMDQLEGQLLAQVLALEKERVALSHSSRRQRQEVEKELDVLQGRVAELEHGSSAYSPPDAFKISIPIRNNYMYARVRKALPELYAFTACMWLRSRSSGTGQGTPFSYSVPGQANEIVLLEAGHEPMELLINDKVAQLPLSLKDNGWHHICIAWTTRDGLWSAYQDGELQGSGENLAAWHPIKPHGILILGQEQDTLGGRFDATQAFVGDIAQFNLWDHALTPAQVLGIANCTAPLLGNVLPWEDKLVEAFGGATKAAFDVCKGRAKA",
# NET1_HUMAN
"MMRAVWEALAALAAVACLVGAVRGGPGLSMFAGQAAQPDPCSDENGHPRRCIPDFVNAAFGKDVRVSSTCGRPPARYCVVSERGEERLRSCHLCNASDPKKAHPPAFLTDLNNPHNLTCWQSENYLQFPHNVTLTLSLGKKFEVTYVSLQFCSPRPESMAIYKSMDYGRTWVPFQFYSTQCRKMYNRPHRAPITKQNEQEAVCTDSHTDMRPLSGGLIAFSTLDGRPSAHDFDNSPVLQDWVTATDIRVAFSRLHTFGDENEDDSELARDSYFYAVSDLQVGGRCKCNGHAARCVRDRDDSLVCDCRHNTAGPECDRCKPFHYDRPWQRATAREANECVACNCNLHARRCRFNMELYKLSGRKSGGVCLNCRHNTAGRHCHYCKEGYYRDMGKPITHRKACKACDCHPVGAAGKTCNQTTGQCPCKDGVTGITCNRCAKGYQQSRSPIAPCIKIPVAPPTTAASSVEEPEDCDSYCKASKGKLKINMKKYCKKDYAVQIHILKADKAGDWWKFTVNIISVYKQGTSRIRRGDQSLWIRSRDIACKCPKIKPLKKYLLLGNAEDSPDQSGIVADKSSLVIQWRDTWARRLRKFQQREKKGKCKKA",
# VATG2_HUMAN
"MASQSQGIQQLLQAEKRAAEKVADARKRKARRLKQAKEEAQMEVEQYRREREHEFQSKQQAAMGSQGNLSAEVEQATRRQVQGMQSSQQRNRERVLAQLLGMVCDVRPQVHPNYRISA",
# SNP29_HUMAN
"MSAYPKSYNPFDDDGEDEGARPAPWRDARDLPDGPDAPADRQQYLRQEVLRRAEATAASTSRSLALMYESEKVGVASSEELARQRGVLERTEKMVDKMDQDLKISQKHINSIKSVFGGLVNYFKSKPVETPPEQNGTLTSQPNNRLKEAISTSKEQEAKYQASHPNLRKLDDTDPVPRGAGSAMSTDAYPKNPHLRAYHQKIDSNLDELSMGLGRLKDIALGMQTEIEEQDDILDRLTTKVDKLDVNIKSTERKVRQL",
# CPNE6_HUMAN
"MSDPEMGWVPEPPTMTLGASRVELRVSCHGLLDRDTLTKPHPCVLLKLYSDEQWVEVERTEVLRSCSSPVFSRVLALEYFFEEKQPLQFHVFDAEDGATSPRNDTFLGSTECTLGQIVSQTKVTKPLLLKNGKTAGKSTITIVAEEVSGTNDYVQLTFRAYKLDNKDLFSKSDPFMEIYKTNEDQSDQLVWRTEVVKNNLNPSWEPFRLSLHSLCSCDVHRPLKFLVYDYDSSGKHDFIGEFTSTFQEMQEGTANPGQEMQWDCINPKYRDKKKNYKSSGTVVLAQCTVEKVHTFLDYIMGGCQISFTVAIDFTASNGDPRSSQSLHCLSPRQPNHYLQALRAVGGICQDYDSDKRFPAFGFGARIPPNFEVSHDFAINFDPENPECEEISGVIASYRRCLPQIQLYGPTNVAPIINRVAEPAQREQSTGQATKYSVLLVLTDGVVSDMAETRTAIVRASRLPMSIIIVGVGNADFSDMRLLDGDDGPLRCPRGVPAARDIVQFVPFRDFKDAAPSALAKCVLAEVPRQVVEYYASQGISPGAPRPCTLATTPSPSP",
# SEM4F_HUMAN
"MPASAARPRPGPGQPTASPFPLLLLAVLSGPVSGRVPRSVPRTSLPISEADSCLTRFAVPHTYNYSVLLVDPASHTLYVGARDTIFALSLPFSGERPRRIDWMVPEAHRQNCRKKGKKEDECHNFVQILAIANASHLLTCGTFAFDPKCGVIDVSRFQQVERLESGRGKCPFEPAQRSAAVMAGGVLYAATVKNYLGTEPIITRAVGRAEDWIRTDTLPSWLNAPAFVAAVALSPAEWGDEDGDDEIYFFFTETSRAFDSYERIKVPRVARVCAGDLGGRKTLQQRWTTFLKADLLCPGPEHGRASSVLQDVAVLRPELGAGTPIFYGIFSSQWEGATISAVCAFRPQDIRTVLNGPFRELKHDCNRGLPVVDNDVPQPRPGECITNNMKLRHFGSSLSLPDRVLTFIRDHPLMDRPVFPADGHPLLVTTDTAYLRVVAHRVTSLSGKEYDVLYLGTEDGHLHRAVRIGAQLSVLEDLALFPEPQPVENMKLYHSWLLVGSRTEVTQVNTTNCGRLQSCSECILAQDPVCAWSFRLDECVAHAGEHRGLVQDIESADVSSLCPKEPGERPVVFEVPVATAAHVVLPCSPSSAWASCVWHQPSGVTALTPRRDGLEVVVTPGAMGAYACECQEGGAAHVVAAYSLVWGSQRDAPSRAHTVGAGLAGFFLGILAASLTLILIGRRQQRRRQRELLARDKVGLDLGAPPSGTTSYSQDPPSPSPEDERLPLALAKRGSGFGGFSPPFLLDPCPSPAHIRLTGAPLATCDETSI",
# AP2A1_HUMAN
"MPAVSKGDGMRGLAVFISDIRNCKSKEAEIKRINKELANIRSKFKGDKALDGYSKKKYVCKLLFIFLLGHDIDFGHMEAVNLLSSNKYTEKQIGYLFISVLVNSNSELIRLINNAIKNDLASRNPTFMCLALHCIANVGSREMGEAFAADIPRILVAGDSMDSVKQSAALCLLRLYKASPDLVPMGEWTARVVHLLNDQHMGVVTAAVSLITCLCKKNPDDFKTCVSLAVSRLSRIVSSASTDLQDYTYYFVPAPWLSVKLLRLLQCYPPPEDAAVKGRLVECLETVLNKAQEPPKSKKVQHSNAKNAILFETISLIIHYDSEPNLLVRACNQLGQFLQHRETNLRYLALESMCTLASSEFSHEAVKTHIDTVINALKTERDVSVRQRAADLLYAMCDRSNAKQIVSEMLRYLETADYAIREEIVLKVAILAEKYAVDYSWYVDTILNLIRIAGDYVSEEVWYRVLQIVTNRDDVQGYAAKTVFEALQAPACHENMVKVGGYILGEFGNLIAGDPRSSPPVQFSLLHSKFHLCSVATRALLLSTYIKFINLFPETKATIQGVLRAGSQLRNADVELQQRAVEYLTLSSVASTDVLATVLEEMPPFPERESSILAKLKRKKGPGAGSALDDGRRDPSSNDINGGMEPTPSTVSTPSPSADLLGLRAAPPPAAPPASAGAGNLLVDVFDGPAAQPSLGPTPEEAFLSELEPPAPESPMALLADPAPAADPGPEDIGPPIPEADELLNKFVCKNNGVLFENQLLQIGVKSEFRQNLGRMYLFYGNKTSVQFQNFSPTVVHPGDLQTQLAVQTKRVAAQVDGGAQVQQVLNIECLRDFLTPPLLSVRFRYGGAPQALTLKLPVTINKFFQPTEMAAQDFFQRWKQLSLPQQEAQKIFKANHPMDAEVTKAKLLGFGSALLDNVDPNPENFVGAGIIQTKALQVGCLLRLEPNAQAQMYRLTLRTSKEPVSRHLCELLAQQF",
# STAU1_HUMAN
"MSQVQVQVQNPSAALSGSQILNKNQSLLSQPLMSIPSTTSSLPSENAGRPIQNSALPSASITSTSAAAESITPTVELNALCMKLGKKPMYKPVDPYSRMQSTYNYNMRGGAYPPRYFYPFPVPPLLYQVELSVGGQQFNGKGKTRQAAKHDAAAKALRILQNEPLPERLEVNGRESEEENLNKSEISQVFEIALKRNLPVNFEVARESGPPHMKNFVTKVSVGEFVGEGEGKSKKISKKNAAIAVLEELKKLPPLPAVERVKPRIKKKTKPIVKPQTSPEYGQGINPISRLAQIQQAKKEKEPEYTLLTERGLPRRREFVMQVKVGNHTAEGTGTNKKVAKRNAAENMLEILGFKVPQAQPTKPALKSEEKTPIKKPGDGRKVTFFEPGSGDENGTSNKEDEFRMPYLSHQQLPAGILPMVPEVAQAVGVSQGHHTKDFTRAAPNPAKATVTAMIARELLYGGTSPTAETILKNNISSGHVPHGPLTRPSEQLDYLSRVQGFQVEYKDFPKNNKNEFVSLINCSSQPPLISHGIGKDVESCHDMAALNILKLLSELDQQSTEMPRTGNGPMSVCGRC",
# LATS1_HUMAN
"MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSDAAKAEHNMSKMSTEDPRQVRNPPKFGTHHKALQEIRNSLLPFANETNSSRSTSEVNPQMLQDLQAAGFDEDMVIQALQKTNNRSIEAAIEFISKMSYQDPRREQMAAAAARPINASMKPGNVQQSVNRKQSWKGSKESLVPQRHGPPLGESVAYHSESPNSQTDVGRPLSGSGISAFVQAHPSNGQRVNPPPPPQVRSVTPPPPPRGQTPPPRGTTPPPPSWEPNSQTKRYSGNMEYVISRISPVPPGAWQEGYPPPPLNTSPMNPPNQGQRGISSVPVGRQPIIMQSSSKFNFPSGRPGMQNGTGQTDFMIHQNVVPAGTVNRQPPPPYPLTAANGQSPSALQTGGSAAPSSYTNGSIPQSMMVPNRNSHNMELYNISVPGLQTNWPQSSSAPAQSSPSSGHEIPTWQPNIPVRSNSFNNPLGNRASHSANSQPSATTVTAITPAPIQQPVKSMRVLKPELQTALAPTHPSWIPQPIQTVQPSPFPEGTASNVTVMPPVAEAPNYQGPPPPYPKHLLHQNPSVPPYESISKPSKEDQPSLPKEDESEKSYENVDSGDKEKKQITTSPITVRKNKKDEERRESRIQSYSPQAFKFFMEQHVENVLKSHQQRLHRKKQLENEMMRVGLSQDAQDQMRKMLCQKESNYIRLKRAKMDKSMFVKIKTLGIGAFGEVCLARKVDTKALYATKTLRKKDVLLRNQVAHVKAERDILAEADNEWVVRLYYSFQDKDNLYFVMDYIPGGDMMSLLIRMGIFPESLARFYIAELTCAVESVHKMGFIHRDIKPDNILIDRDGHIKLTDFGLCTGFRWTHDSKYYQSGDHPRQDSMDFSNEWGDPSSCRCGDRLKPLERRAARQHQRCLAHSLVGTPNYIAPEVLLRTGYTQLCDWWSVGVILFEMLVGQPPFLAQTPLETQMKVINWQTSLHIPPQAKLSPEASDLIIKLCRGPEDRLGKNGADEIKAHPFFKTIDFSSDLRQQSASYIPKITHPTDTSNFDPVDPDKLWSDDNEEENVNDTLNGWYKNGKHPEHAFYEFTFRRFFDDNGYPYNYPKPIEYEYINSQGSEQQSDEDDQNTGSEIKNRDLVYV",
# DLGP3_HUMAN
"MRGYHGDRGSHPRPARFADQQHMDVGPAARAPYLLGSREAFSTEPRFCAPRAGLGHISPEGPLSLSEGPSVGPEGGPAGAGVGGGSSTFPRMYPGQGPFDTCEDCVGHPQGKGAPRLPPTLLDQFEKQLPVQQDGFHTLPYQRGPAGAGPGPAPGTGTAPEPRSESPSRIRHLVHSVQKLFAKSHSLEAPGKRDYNGPKAEGRGGSGGDSYPGPGSGGPHTSHHHHHHHHHHHHQSRHGKRSKSKDRKGDGRHQAKSTGWWSSDDNLDSDSGFLAGGRPPGEPGGPFCLEGPDGSYRDLSFKGRSGGSEGRCLACTGMSMSLDGQSVKRSAWHTMMVSQGRDGYPGAGPGKGLLGPETKAKARTYHYLQVPQDDWGGYPTGGKDGEIPCRRMRSGSYIKAMGDEESGDSDGSPKTSPKAVARRFTTRRSSSVDQARINCCVPPRIHPRSSIPGYSRSLTTGQLSDELNQQLEAVCGSVFGELESQAVDALDLPGCFRMRSHSYLRAIQAGCSQDDDCLPLLATPAAVSGRPGSSFNFRKAPPPIPPGSQAPPRISITAQSSTDSAHESFTAAEGPARRCSSADGLDGPAMGARTLELAPVPPRASPKPPTLIIKTIPGREELRSLARQRKWRPSIGVQVETISDSDTENRSRREFHSIGVQVEEDKRRARFKRSNSVTAGVQADLELEGLAGLATVATEDKALQFGRSFQRHASEPQPGPRAPTYSVFRTVHTQGQWAYREGYPLPYEPPATDGSPGPAPAPTPGPGAGRRDSWIERGSRSLPDSGRASPCPRDGEWFIKMLRAEVEKLEHWCQQMEREAEDYELPEEILEKIRSAVGSTQLLLSQKVQQFFRLCQQSMDPTAFPVPTFQDLAGFWDLLQLSIEDVTLKFLELQQLKANSWKLLEPKEEKKVPPPIPKKPLRGRGVPVKERSLDSVDRQRQEARKRLLAAKRAASFRHSSATESADSIEIYIPEAQTRL",
# NOE2_HUMAN
"MWPLTVPPPLLLLLCSGLAGQTLFQNPEEGWQLYTSAQAPDGKCICTAVIPAQSTCSRDGRSRELRQLMEKVQNVSQSMEVLELRTYRDLQYVRGMETLMRSLDARLRAADGSLSAKSFQELKDRMTELLPLSSVLEQYKADTRTIVRLREEVRNLSGSLAAIQEEMGAYGYEDLQQRVMALEARLHACAQKLGCGKLTGVSNPITVRAMGSRFGSWMTDTMAPSADSRVWYMDGYYKGRRVLEFRTLGDFIKGQNFIQHLLPQPWAGTGHVVYNGSLFYNKYQSNVVVKYHFRSRSVLVQRSLPGAGYNNTFPYSWGGFSDMDFMVDESGLWAVYTTNQNAGNIVVSRLDPHTLEVMRSWDTGYPKRSAGEAFMICGVLYVTNSHLAGAKVYFAYFTNTSSYEYTDVPFHNQYSHISMLDYNPRERALYTWNNGHQVLYNVTLFHVISTSGDP",
# LGI1_HUMAN
"MESERSKRMGNACIPLKRIAYFLCLLSALLLTEGKKPAKPKCPAVCTCTKDNALCENARSIPRTVPPDVISLSFVRSGFTEISEGSFLFTPSLQLLLFTSNSFDVISDDAFIGLPHLEYLFIENNNIKSISRHTFRGLKSLIHLSLANNNLQTLPKDIFKGLDSLTNVDLRGNSFNCDCKLKWLVEWLGHTNATVEDIYCEGPPEYKKRKINSLSSKDFDCIITEFAKSQDLPYQSLSIDTFSYLNDEYVVIAQPFTGKCIFLEWDHVEKTFRNYDNITGTSTVVCKPIVIETQLYVIVAQLFGGSHIYKRDSFANKFIKIQDIEILKIRKPNDIETFKIENNWYFVVADSSKAGFTTIYKWNGNGFYSHQSLHAWYRDTDVEYLEIVRTPQTLRTPHLILSSSSQRPVIYQWNKATQLFTNQTDIPNMEDVYAVKHFSVKGDVYICLTRFIGDSKVMKWGGSSFQDIQRMPSRGSMVFQPLQINNYQYAILGSDYSFTQVYNWDAEKAKFVKFQELNVQAPRSFTHVSINKRNFLFASSFKGNTQIYKHVIVDLSA",
# S1PR4_HUMAN
"MNATGTPVAPESCQQLAAGGHSRLIVLHYNHSGRLAGRGGPEDGGLGALRGLSVAASCLVVLENLLVLAAITSHMRSRRWVYYCLVNITLSDLLTGAAYLANVLLSGARTFRLAPAQWFLREGLLFTALAASTFSLLFTAGERFATMVRPVAESGATKTSRVYGFIGLCWLLAALLGMLPLLGWNCLCAFDRCSSLLPLYSKRYILFCLVIFAGVLATIMGLYGAIFRLVQASGQKAPRPAARRKARRLLKTVLMILLAFLVCWGPLFGLLLADVFGSNLWAQEYLRGMDWILALAVLNSAVNPIIYSFRSREVCRAVLSFLCCGCLRLGMRGPGDCLARAVEAHSGASTTDSSLRPRDSFRGSRSLSFRMREPLSSISSVRSI",
# NUDT3_HUMAN
"MMKLKSNQTRTYDGDGYKKRAACLCFRSESEEEVLLVSSSRHPDRWIVPGGGMEPEEEPSVAAVREVCEEAGVKGTLGRLVGIFENQERKHRTYVYVLIVTEVLEDWEDSVNIGRKREWFKIEDAIKVLQYHKPVQASYFETLRQGYSANNGTPVVATTYSVSAQSSMSGIR",
# F107A_HUMAN
"MYSEIQRERADIGGLMARPEYREWNPELIKPKKLLNPVKASRSHQELHRELLMNHRRGLGVDSKPELQRVLEHRRRNQLIKKKKEELEAKRLQCPFEQELLRRQQRLNQLEKPPEKEEDHAPEFIKVRENLRRIATLTSEEREL",
# APCL_HUMAN
"MASSVAPYEQLVRQVEALKAENSHLRQELRDNSSHLSKLETETSGMKEVLKHLQGKLEQEARVLVSSGQTEVLEQLKALQMDITSLYNLKFQPPTLGPEPAARTPEGSPVHGSGPSKDSFGELSRATIRLLEELDRERCFLLNEIEKEEKEKLWYYSQLQGLSKRLDELPHVETQFSMQMDLIRQQLEFEAQHIRSLMEERFGTSDEMVQRAQIRASRLEQIDKELLEAQDRVQQTEPQALLAVKSVPVDEDPETEVPTHPEDGTPQPGNSKVEVVFWLLSMLATRDQEDTARTLLAMSSSPESCVAMRRSGCLPLLLQILHGTEAAAGGRAGAPGAPGAKDARMRANAALHNIVFSQPDQGLARKEMRVLHVLEQIRAYCETCWDWLQARDGGPEGGGAGSAPIPIEPQICQATCAVMKLSFDEEYRRAMNELGGLQAVAELLQVDYEMHKMTRDPLNLALRRYAGMTLTNLTFGDVANKATLCARRGCMEAIVAQLASDSEELHQVVSSILRNLSWRADINSKKVLREAGSVTALVQCVLRATKESTLKSVLSALWNLSAHSTENKAAICQVDGALGFLVSTLTYKCQSNSLAIIESGGGILRNVSSLVATREDYRQVLRDHNCLQTLLQHLTSHSLTIVSNACGTLWNLSARSARDQELLWDLGAVGMLRNLVHSKHKMIAMGSAAALRNLLAHRPAKHQAAATAVSPGSCVPSLYVRKQRALEAELDARHLAQALEHLEKQGPPAAEAATKKPLPPLRHLDGLAQDYASDSGCFDDDDAPSSLAAAAATGEPASPAALSLFLGSPFLQGQALARTPPTRRGGKEAEKDTSGEAAVAAKAKAKLALAVARIDQLVEDISALHTSSDDSFSLSSGDPGQEAPREGRAQSCSPCRGPEGGRREAGSRAHPLLRLKAAHASLSNDSLNSGSASDGYCPREHMLPCPLAALASRREDPRCGQPRPSRLDLDLPGCQAEPPAREATSADARVRTIKLSPTYQHVPLLEGASRAGAEPLAGPGISPGARKQAWLPADHLSKVPEKLAAAPLSVASKALQKLAAQEGPLSLSRCSSLSSLSSAGRPGPSEGGDLDDSDSSLEGLEEAGPSEAELDSTWRAPGATSLPVAIPAPRRNRGRGLGVEDATPSSSSENYVQETPLVLSRCSSVSSLGSFESPSIASSIPSEPCSGQGSGTISPSELPDSPGQTMPPSRSKTPPLAPAPQGPPEATQFSLQWESYVKRFLDIADCRERCRLPSELDAGSVRFTVEKPDENFSCASSLSALALHEHYVQQDVELRLLPSACPERGGGAGGAGLHFAGHRRREEGPAPTGSRPRGAADQELELLRECLGAAVPARLRKVASALVPGRRALPVPVYMLVPAPAPAQEDDSCTDSAEGTPVNFSSAASLSDETLQGPPRDQPGGPAGRQRPTGRPTSARQAMGHRHKAGGAGRSAEQSRGAGKNRAGLELPLGRPPSAPADKDGSKPGRTRGDGALQSLCLTTPTEEAVYCFYGNDSDEEPPAAAPTPTHRRTSAIPRAFTRERPQGRKEAPAPSKAAPAAPPPARTQPSLIADETPPCYSLSSSASSLSEPEPSEPPAVHPRGREPAVTKDPGPGGGRDSSPSPRAAEELLQRCISSALPRRRPPVSGLRRRKPRATRLDERPAEGSRERGEEAAGSDRASDLDSVEWRAIQEGANSIVTWLHQAAAATREASSESDSILSFVSGLSVGSTLQPPKHRKGRQAEGEMGSARRPEKRGAASVKTSGSPRSPAGPEKPRGTQKTTPGVPAVLRGRTVIYVPSPAPRAQPKGTPGPRATPRKVAPPCLAQPAAPAKVPSPGQQRSRSLHRPAKTSELATLSQPPRSATPPARLAKTPSSSSSQTSPASQPLPRKRPPVTQAAGALPGPGASPVPKTPARTLLAKQHKTQRSPVRIPFMQRPARRGPPPLARAVPEPGPRGRAGTEAGPGARGGRLGLVRVASALSSGSESSDRSGFRRQLTFIKESPGLRRRRSELSSAESAASAPQGASPRRGRPALPAVFLCSSRCEELRAAPRQGPAPARQRPPAARPSPGERPARRTTSESPSRLPVRAPAARPETVKRYASLPHISVARRPDGAVPAAPASADAARRSSDGEPRPLPRVAAPGTTWRRIRDEDVPHILRSTLPATALPLRGSTPEDAPAGPPPRKTSDAVVQTEEVAAPKTNSSTSPSLETREPPGAPAGGQLSLLGSDVDGPSLAKAPISAPFVHEGLGVAVGGFPASRHGSPSRSARVPPFNYVPSPMVVAATTDSAAEKAPATASATLLE",
# CLPT1_HUMAN
"MAAAQEADGARSAVVAAGGGSSGQVTSNGSIGRDPPAETQPQNPPAQPAPNAWQVIKGVLFRIFIIWAISSWFRRGPAPQDQAGPGGAPRVASRNLFPKDTLMNLHVYISEHEHFTDFNATSALFWEQHDLVYGDWTSGENSDGCYEHFAELDIPQSVQQNGSIYIHVYFTKSGFHPDPRQKALYRRLATVHMSRMINKYKRRRFQKTKNLLTGETEADPEMIKRAEDYGPVEVISHWHPNITINIVDDHTPWVKGSVPPPLDQYVKFDAVSGDYYPIIYFNDYWNLQKDYYPINESLASLPLRVSFCPLSLWRWQLYAAQSTKSPWNFLGDELYEQSDEEQDSVKVALLETNPYLLALTIIVSIVHSVFEFLAFKNDIQFWNSRQSLEGLSVRSVFFGVFQSFVVLLYILDNETNFVVQVSVFIGVLIDLWKITKVMDVRLDREHRVAGIFPRLSFKDKSTYIESSTKVYDDMAFRYLSWILFPLLGCYAVYSLLYLEHKGWYSWVLSMLYGFLLTFGFITMTPQLFINYKLKSVAHLPWRMLTYKALNTFIDDLFAFVIKMPVMYRIGCLRDDVVFFIYLYQRWIYRVDPTRVNEFGMSGEDPTAAAPVAEVPTAAGALTPTPAPTTTTATREEASTSLPTKPTQGASSASEPQEAPPKPAEDKKKD",
# NAPSA_HUMAN
"MSPPPLLQPLLLLLPLLNVEPSGATLIRIPLHRVQPGRRILNLLRGWREPAELPKLGAPSPGDKPIFVPLSNYRDVQYFGEIGLGTPPQNFTVAFDTGSSNLWVPSRRCHFFSVPCWLHHRFDPKASSSFQANGTKFAIQYGTGRVDGILSEDKLTIGGIKGASVIFGEALWEPSLVFAFAHFDGILGLGFPILSVEGVRPPMDVLVEQGLLDKPVFSFYLNRDPEEPDGGELVLGGSDPAHYIPPLTFVPVTVPAYWQIHMERVKVGPGLTLCAKGCAAILDTGTSLITGPTEEIRALHAAIGGIPLLAGEYIILCSEIPKLPAVSFLLGGVWFNLTAHDYVIQTTRNGVRLCLSGFQALDVPPPAGPFWILGDVFLGTYVAVFDRGDMKSSARVGLARARTRGADLGWGETAQAQFPG",
# ABL1_HUMAN
"MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSSTAFIPLISTRVSLRKTRQPPERIASGAITKGVVLDSTEALCLAISRNSEQMASHSAVLEAGKNLYTFCVSYVDSIQQMRNKFAFREAINKLENNLRELQICPATAGSGPAATQDFSKLLSSVKEISDIVQR",
# EGFR_HUMAN
"MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRALMDEEDMDDVVDADEYLIPQQGFFSSPSTSRTPLLSSLSATSNNSTVACIDRNGLQSCPIKEDSFLQRYSSDPTGALTEDSIDDTFLPVPEYINQSVPKRPAGSVQNPVYHNQPLNPAPSRDPHYQDPHSTAVGNPEYLNTVQPTCVNSTFDSPAHWAQKGSHQISLDNPDYQQDFFPKEAKPNGIFKGSTAENAEYLRVAPQSSEFIGA",
# PLMN_HUMAN
"MEHKEVVLLLLLFLKSGQGEPLDDYVNTQGASLFSVTKKQLGAGSIEECAAKCEEDEEFTCRAFQYHSKEQQCVIMAENRKSSIIIRMRDVVLFEKKVYLSECKTGNGKNYRGTMSKTKNGITCQKWSSTSPHRPRFSPATHPSEGLEENYCRNPDNDPQGPWCYTTDPEKRYDYCDILECEEECMHCSGENYDGKISKTMSGLECQAWDSQSPHAHGYIPSKFPNKNLKKNYCRNPDRELRPWCFTTDPNKRWELCDIPRCTTPPPSSGPTYQCLKGTGENYRGNVAVTVSGHTCQHWSAQTPHTHNRTPENFPCKNLDENYCRNPDGKRAPWCHTTNSQVRWEYCKIPSCDSSPVSTEQLAPTAPPELTPVVQDCYHGDGQSYRGTSSTTTTGKKCQSWSSMTPHRHQKTPENYPNAGLTMNYCRNPDADKGPWCFTTDPSVRWEYCNLKKCSGTEASVVAPPPVVLLPDVETPSEEDCMFGNGKGYRGKRATTVTGTPCQDWAAQEPHRHSIFTPETNPRAGLEKNYCRNPDGDVGGPWCYTTNPRKLYDYCDVPQCAAPSFDCGKPQVEPKKCPGRVVGGCVAHPHSWPWQVSLRTRFGMHFCGGTLISPEWVLTAAHCLEKSPRPSSYKVILGAHQEVNLEPHVQEIEVSRLFLEPTRKDIALLKLSSPAVITDKVIPACLPSPNYVVADRTECFITGWGETQGTFGAGLLKEAQLPVIENKVCNRYEFLNGRVQSTELCAGHLAGGTDSCQGDSGGPLVCFEKDKYILQGVTSWGLGCARPNKPGVYVRVSRFVTWIEGVMRNN",
# TPA_HUMAN
"MDAMKRGLCCVLLLCGAVFVSPSQEIHARFRRGARSYQVICRDEKTQMIYQQHQSWLRPVLRSNRVEYCWCNSGRAQCHSVPVKSCSEPRCFNGGTCQQALYFSDFVCQCPEGFAGKCCEIDTRATCYEDQGISYRGTWSTAESGAECTNWNSSALAQKPYSGRRPDAIRLGLGNHNYCRNPDRDSKPWCYVFKAGKYSSEFCSTPACSEGNSDCYFGNGSAYRGTHSLTESGASCLPWNSMILIGKVYTAQNPSAQALGLGKHNYCRNPDGDAKPWCHVLKNRRLTWEYCDVPSCSTCGLRQYSQPQFRIKGGLFADIASHPWQAAIFAKHRRSPGERFLCGGILISSCWILSAAHCFQERFPPHHLTVILGRTYRVVPGEEEQKFEVEKYIVHKEFDDDTYDNDIALLQLKSDSSRCAQESSVVRTVCLPPADLQLPDWTECELSGYGKHEALSPFYSERLKEAHVRLYPSSRCTSQHLLNRTVTDNMLCAGDTRSGGPQANLHDACQGDSGGPLVCLNDGRMTLVGIISWGLGCGQKDVPGVYTKVTNYLDWIRDNMRP",
# RASH_HUMAN
"MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHQYREQIKRVKDSDDVPMVLVGNKCDLAARTVESRQAQDLARSYGIPYIETSAKTRQGVEDAFYTLVREIRQHKLRKLNPPDESGPGCMSCKCVLS",
# ANF_HUMAN
"MSSFSTTTVSFLLLLAFQLLGQTRANPMYNAVSNADLMDFKNLLDHLEEKMPLEDEVVPPQVLSEPNEEAGAALSPLPEVPPWTGEVSPAQRDGGALGRGPWDSSDRSALLKSKLRALLTAPRSLRRSSCFGGRMDRIGAQSGLGCNSFRY",
# NEU1_HUMAN
"MAGPSLACCLLGLLALTSACYIQNCPLGGKRAAPDLDVRKCLPCGPGGKGRCFGPNICCAEELGCFVGTAEALRCQEENYLPSPCQSGQKACGSGGRCAVLGLCCSPDGCHADPACDAEATFSQR",
# NEU2_HUMAN
"MPDTMLPACFLGLLAFSSACYFQNCPRGGKRAMSDLELRQCLPCGPGGKGRCFGPSICCADELGCFVGTAEALRCQEENYLPSPCQSGQKACGSGGRCAAFGVCCNDESCVTEPECREGFHRRARASDRSNATQLDGPAGALLLRLVQLAGAPEPFEPAQPDAY",
# PENK_HUMAN
"MARFLTLCTWLLLLGPGLLATVRAECSQDCATCSYRLVRPADINFLACVMECEGKLPSLKIWETCKELLQLSKPELPQDGTSTLRENSKPEESHLLAKRYGGFMKRYGGFMKKMDELYPMEPEEEANGSEILAKRYGGFMKKDAEEDDSLANSSDLLKELLETGDNRERSHHQDGSDNEEEVSKRYGGFMRGLKRSPQLEDEAKELQKRYGGFMRRVGRPEWWMDYQKRYGGFLKRFAEALPSDEEGESYSKEVPEMEKRYGGFMRF",
# PDYN_HUMAN
"MAWQGLVLAACLLMFPSTTADCLSRCSLCAVKTQDGPKPINPLICSLQCQAALLPSEEWERCQSFLSFFTPSTLGLNDKEDLGSKSVGEGPYSELAKLSGSFLKELEKSKFLPSISTKENTLSKSLEEKLRGLSDGFREGAESELMRDAQLNDGAMETGTLYLAEEDPKEQVKRYGGFLRKYPKRSSEVAGEGDGDSMGHEDLYKRYGGFLRRIRPKLKWDNQKRYGGFLRRQFKVVTRSQEDPNAYSGELFDA",
# CALC_HUMAN
"MGFQKFSPFLALSILVLLQAGSLHAAPFRSALESSPADPATLSEDEARLLLAALVQDYVQMKASELEQEQEREGSSLDSPRSKRCGNLSTCMLGTYTQDFNKFHTFPQTAIGVGAPGKKRDMSSDLERDHRPHVSMPQNAN",
# NPY_HUMAN
"MLGNKRLGLSGLTLALSLLVCLGALAEAYPSKPDNPGEDAPAEDMARYYSALRHYINLITRQRYGKRSSPETLISDLLMRESTENVPRTRLEDPAMW",
# APOE_HUMAN
"MKVLWAALLVTFLAGCQAKVEQAVETEPEPELRQQTEWQSGQRWELALGRFWDYLRWVQTLSEQVQEELLSSQVTQELRALMDETMKELKAYKSELEEQLTPVAEETRARLSKELQAAQARLGADMEDVCGRLVQYRGEVQAMLGQSTEELRVRLASHLRKLRKRLLRDADDLQKRLAVYQAGAREGAERGLSAIRERLGPLVEQGRVRAATVGSLAGQPLQERAQAWGERLRARMEEMGSRTRDRLDEVKEQVAEVRAKLEEQAQQIRLQAEAFQARLKSWFEPLVEDMQRQWAGLVEKVQAAVGTSAAPVPSDNH",
# APOC2_HUMAN
"MGTRLLPALFLVLLVLGFEVQGTQQPQQDEMPSPTFLTQVKESLSSYWESAKTAAQNLYEKTYLPAVDEKLRDLYSKSTAAMSTYTGIFTDQVLSVLKGEE",
# FIBA_HUMAN
"MFSMRIVCLVLSVVGTAWTADSGEGDFLAEGGGVRGPRVVERHQSACKDSDWPFCSDEDWNYKCPSGCRMKGLIDEVNQDFTNRINKLKNSLFEYQKNNKDSHSLTTNIMEILRGDFSSANNRDNTYNRVSEDLRSRIEVLKRKVIEKVQHIQLLQKNVRAQLVDMKRLEVDIDIKIRSCRGSCSRALAREVDLKDYEDQQKQLEQVIAKDLLPSRDRQHLPLIKMKPVPDLVPGNFKSQLQKVPPEWKALTDMPQMRMELERPGGNEITRGGSTSYGTGSETESPRNPSSAGSWNSGSSGPGSTGNRNPGSSGTGGTATWKPGSSGPGSTGSWNSGSSGTGSTGNQNPGSPRPGSTGTWNPGSSERGSAGHWTSESSVSGSTGQWHSESGSFRPDSPGSGNARPNNPDWGTFEEVSGNVSPGTRREYHTEKLVTSKGDKELRTGKEKVTSGSTTTTRRSCSKTVTKTVIGPDGHKEVTKEVVTSEDGSDCPEAMDLGTLSGIGTLDGFRHRHPDEAAFFDTASTGKTFPGFFSPMLGEFVSETESRGSESGIFTNTKESSSHHPGIAEFPSRGKSSSYSKQFTSSTSYNRGDSTFESKSYKMADEAGSEADHEGTHSTKRGHAKSRPVRDCDDVLQTHPSGTQSGIFNIKLPGSSKIFSVYCDQETSLGGWLLIQQRMDGSLNFNRTWQDYKRGFGSLNDEGEGEFWLGNDYLHLLTQRGSVLRVELEDWAGNEAYAEYHFRVGSEAEGYALQVSSYEGTAGDALIEGSVEEGAEYTSHNNMQFSTFDRDADQWEENCAEVYGGGWWYNNCQAANLNGIYYPGGSYDPRNNSPYEIENGVVWVSFRGADYSLRAVRMKIRPLVTQ",
# FIBB_HUMAN
"MKRMVSWSFHKLKTMKHLLLLLLCVFLVKSQGVNDNEEGFFSARGHRPLDKKREEAPSLRPAPPPISGGGYRARPAKAAATQKKVERKAPDAGGCLHADPDLGVLCPTGCQLQEALLQQERPIRNSVDELNNNVEAVSQTSSSSFQYMYLLKDLWQKRQKQVKDNENVVNEYSSELEKHQLYIDETVNSNIPTNLRVLRSILENLRSKIQKLESDVSAQMEYCRTPCTVSCNIPVVSGKECEEIIRKGGETSEMYLIQPDSSVKPYRVYCDMNTENGGWTVIQNRQDGSVDFGRKWDPYKQGFGNVATNTDGKNYCGLPGEYWLGNDKISQLTRMGPTELLIEMEDWKGDKVKAHYGGFTVQNEANKYQISVNKYRGTAGNALMDGASQLMGENRTMTIHNGMFFSTYDRDNDGWLTSDPRKQCSKEDGGGWWYNRCHAANPNGRYYWGGQYTWDMAKHGTDDGVVWMNWKGSWYSMRKMSMKIRPFFPQQ",
# FIBG_HUMAN
"MSWSLHPRNLILYFYALLFLSSTCVAYVATRDNCCILDERFGSYCPTTCGIADFLSTYQTKVDKDLQSLEDILHQVENKTSEVKQLIKAIQLTYNPDESSKPNMIDAATLKSRKMLEEIMKYEASILTHDSSIRYLQEIYNSNNQKIVNLKEKVAQLEAQCQEPCKDTVQIHDITGKDCQDIANKGAKQSGLYFIKPLKANQQFLVYCEIDGSGNGWTVFQKRLDGSVDFKKNWIQYKEGFGHLSPTGTTEFWLGNEKIHLISTQSAIPYALRVELEDWNGRTSTADYAMFKVGPEADKYRLTYAYFAGGDAGDAFDGFDFGDDPSDKFFTSHNGMQFSTWDNDNDKFEGNCAEQDGSGWWMNKCHAGHLNGVYYQGGTYSKASTPNGYDNGIIWATWKTRWYSMKKTTMKIIPFNRLTIGEGQQHHLGGAKQVRPEHPAETEYDSLYPEDDL",
# ACHA_HUMAN
"MEPWPLLLLFSLCSAGLVLGSEHETRLVAKLFKDYSSVVRPVEDHRQVVEVTVGLQLIQLINVDEVNQIVTTNVRLKQQWVDYNLKWNPDDYGGVKKIHIPSEKIWRPDLVLYNNADGDFAIVKFTKVLLQYTGHITWTPPAIFKSYCEIIVTHFPFDEQNCSMKLGTWTYDGSVVAINPESDQPDLSNFMESGEWVIKESRGWKHSVTYSCCPDTPYLDITYHFVMQRLPLYFIVNVIIPCLLFSFLTGLVFYLPTDSGEKMTLSISVLLSLTVFLLVIVELIPSTSSAVPLIGKYMLFTMVFVIASIIITVIVINTHHRSPSTHVMPNWVRKVFIDTIPNIMFFSTMKRPSREKQDKKIFTEDIDISDISGKPGPPPMGFHSPLIKHPEVKSAIEGIKYIAETMKSDQESNNAAAEWKYVAMVMDHILLGVFMLVCIIGTLAVFAGRLIELNQQG",
# C1QA_HUMAN
"MEGPRGWLVLCVLAISLASMVTEDLCRAPDGKKGEAGRPGRRGRPGLKGEQGEPGAPGIRTGIQGLKGDQGEPGPSGNPGKVGYPGPSGPLGARGIPGIKGTKGSPGNIKDQPRPAFSAIRRNPPMGGNVVIFDTVITNQEEPYQNHSGRFVCTVPGYYYFTFQVLSQWEICLSIVSSSRGQVRRSLGFCDTTNKGLFQVVSGGMVLQLQQGDQVWVEKDPKKGHIYQGSEADSVFSGFLIFPSA",
# C1QB_HUMAN
"MMMKIPWGSIPVLMLLLLLGLIDISQAQLSCTGPPAIPGIPGIPGTPGPDGQPGTPGIKGEKGLPGLAGDHGEFGEKGDPGIPGNPGKVGPKGPMGPKGGPGAPGAPGPKGESGDYKATQKIAFSATRTINVPLRRDQTIRFDHVITNMNNNYEPRSGKFTCKVPGLYYFTYHASSRGNLCVNLMRGRERAQKVVTFCDYAYNTFQVTTGGMVLKLEQGENVFLQATDKNSLLGMEGANSIFSGFLLFPDMEA",
# C1QC_HUMAN
"MDVGPSSLPHLGLKLLLLLLLLPLRGQANTGCYGIPGMPGLPGAPGKDGYDGLPGPKGEPGIPAIPGIRGPKGQKGEPGLPGHPGKNGPMGPPGMPGVPGPMGIPGEPGEEGRYKQKFQSVFTVTRQTHQPPAPNSLIRFNAVLTNPQGDYDTSTGKFTCKVPGLYYFVYHASHTANLCVLLYRSGVKVVTFCGHTSKTNQVNSGGVLLRLQVGEEVWLAVNDYYDMVGIQGSDSVFSGFLLFPD",
# TFR1_HUMAN
"MMDQARSAFSNLFGGEPLSYTRFSLARQVDGDNSHVEMKLAVDEEENADNNTKANVTKPKRCSGSICYGTIAVIVFFLIGFMIGYLGYCKGVEPKTECERLAGTESPVREEPGEDFPAARRLYWDDLKRKLSEKLDSTDFTGTIKLLNENSYVPREAGSQKDENLALYVENQFREFKLSKVWRDQHFVKIQVKDSAQNSVIIVDKNGRLVYLVENPGGYVAYSKAATVTGKLVHANFGTKKDFEDLYTPVNGSIVIVRAGKITFAEKVANAESLNAIGVLIYMDQTKFPIVNAELSFFGHAHLGTGDPYTPGFPSFNHTQFPPSRSSGLPNIPVQTISRAAAEKLFGNMEGDCPSDWKTDSTCRMVTSESKNVKLTVSNVLKEIKILNIFGVIKGFVEPDHYVVVGAQRDAWGPGAAKSGVGTALLLKLAQMFSDMVLKDGFQPSRSIIFASWSAGDFGSVGATEWLEGYLSSLHLKAFTYINLDKAVLGTSNFKVSASPLLYTLIEKTMQNVKHPVTGQFLYQDSNWASKVEKLTLDNAAFPFLAYSGIPAVSFCFCEDTDYPYLGTTMDTYKELIERIPELNKVARAAAEVAGQFVIKLTHDVELNLDYERYNSQLLSFVRDLNQYRADIKEMGLSLQWLYSARGDFFRATSRLTTDFGNAEKTDRFVMKKLNDRVMRVEYHFLSPYVSPKESPFRHVFWGSGSHTLPALLENLKLRKQNNGAFNETLFRNQLALATWTIQGAANALSGDVWDIDNEF",
# ANXA1_HUMAN
"MAMVSEFLKQAWFIENEEQEYVQTVKSSKGGPGSAVSPYPTFNPSSDVAALHKAIMVKGVDEATIIDILTKRNNAQRQQIKAAYLQETGKPLDETLKKALTGHLEEVVLALLKTPAQFDADELRAAMKGLGTDEDTLIEILASRTNKEIRDINRVYREELKRDLAKDITSDTSGDFRNALLSLAKGDRSEDFGVNEDLADSDARALYEAGERRKGTDVNVFNTILTTRSYPQLRRVFQKYTKYSKHDMNKVLDLELKGDIEKCLTAIVKCATSKPAFFAEKLHQAMKGVGTRHKALIRIMVSRSEIDMNDIKAFYQKMYGISLCQAILDETKGDYEKILVALCGGN",
# GCR_HUMAN
"MDSKESLTPGREENPSSVLAQERGDVMDFYKTLRGGATVKVSASSPSLAVASQSDSKQRRLLVDFPKGSVSNAQQPDLSKAVSLSMGLYMGETETKVMGNDLGFPQQGQISLSSGETDLKLLEESIANLNRSTSVPENPKSSASTAVSAAPTEKEFPKTHSDVSSEQQHLKGQTGTNGGNVKLYTTDQSTFDILQDLEFSSGSPGKETNESPWRSDLLIDENCLLSPLAGEDDSFLLEGNSNEDCKPLILPDTKPKIKDNGDLVLSSPSNVTLPQVKTEKEDFIELCTPGVIKQEKLGTVYCQASFPGANIIGNKMSAISVHGVSTSGGQMYHYDMNTASLSQQQDQKPIFNVIPPIPVGSENWNRCQGSGDDNLTSLGTLNFPGRTVFSNGYSSPSMRPDVSSPPSSSSTATTGPPPKLCLVCSDEASGCHYGVLTCGSCKVFFKRAVEGQHNYLCAGRNDCIIDKIRRKNCPACRYRKCLQAGMNLEARKTKKKIKGIQQATTGVSQETSENPGNKTIVPATLPQLTPTLVSLLEVIEPEVLYAGYDSSVPDSTWRIMTTLNMLGGRQVIAAVKWAKAIPGFRNLHLDDQMTLLQYSWMFLMAFALGWRSYRQSSANLLCFAPDLIINEQRMTLPCMYDQCKHMLYVSSELHRLQVSYEEYLCMKTLLLLSSVPKDGLKSQELFDEIRMTYIKELGKAIVKREGNSSQNWQRFYQLTKLLDSMHEVVENLLNYCFQTFLDKTMSIEFPEMLAEIITNQIPKYSNGNIKKLLFHQK",
# PRIO_HUMAN
"MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGGNRYPPQGGGGWGQPHGGGWGQPHGGGWGQPHGGGWGQPHGGGWGQGGGTHSQWNKPSKPKTNMKHMAGAAAAGAVVGGLGGYMLGSAMSRPIIHFGSDYEDRYYRENMHRYPNQVYYRPMDEYSNQNNFVHDCVNITIKQHTVTTTTKGENFTETDVKMMERVVEQMCITQYERESQAYYQRGSSMVLFSSPPVILLISFLIFLIVG",
# THY1_HUMAN
"MNLAISIALLLTVLQVSRGQKVTSLTACLVDQSLRLDCRHENTSSSPIQYEFSLTRETKKHVLFGTVGVPEHTYRSRTNFTSKYNMKVLYLSAFTSKDEGTYTCALHHSGHSPPISSQNVTVLRDKLVKCEGISLLAQNTSWLLLLLLSLSLLQATDFMSL",
# G3P_HUMAN
"MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHGKFHGTVKAENGKLVINGNPITIFQERDPSKIKWGDAGAEYVVESTGVFTTMEKAGAHLQGGAKRVIISAPSADAPMFVMGVNHEKYDNSLKIISNASCTTNCLAPLAKVIHDNFGIVEGLMTTVHAITATQKTVDGPSGKLWRDGRGALQNIIPASTGAAKAVGKVIPELNGKLTGMAFRVPTANVSVVDLTCRLEKPAKYDDIKKVVKQASEGPLKGILGYTEHQVVSSDFNSDTHSSTFDAGAGIALNDHFVKLISWYDNEFGYSNRVVDLMAHMASKE",
# ERBB2_HUMAN
"MELAALCRWGLLLALLPPGAASTQVCTGTDMKLRLPASPETHLDMLRHLYQGCQVVQGNLELTYLPTNASLSFLQDIQEVQGYVLIAHNQVRQVPLQRLRIVRGTQLFEDNYALAVLDNGDPLNNTTPVTGASPGGLRELQLRSLTEILKGGVLIQRNPQLCYQDTILWKDIFHKNNQLALTLIDTNRSRACHPCSPMCKGSRCWGESSEDCQSLTRTVCAGGCARCKGPLPTDCCHEQCAAGCTGPKHSDCLACLHFNHSGICELHCPALVTYNTDTFESMPNPEGRYTFGASCVTACPYNYLSTDVGSCTLVCPLHNQEVTAEDGTQRCEKCSKPCARVCYGLGMEHLREVRAVTSANIQEFAGCKKIFGSLAFLPESFDGDPASNTAPLQPEQLQVFETLEEITGYLYISAWPDSLPDLSVFQNLQVIRGRILHNGAYSLTLQGLGISWLGLRSLRELGSGLALIHHNTHLCFVHTVPWDQLFRNPHQALLHTANRPEDECVGEGLACHQLCARGHCWGPGPTQCVNCSQFLRGQECVEECRVLQGLPREYVNARHCLPCHPECQPQNGSVTCFGPEADQCVACAHYKDPPFCVARCPSGVKPDLSYMPIWKFPDEEGACQPCPINCTHSCVDLDDKGCPAEQRASPLTSIISAVVGILLVVVLGVVFGILIKRRQQKIRKYTMRRLLQETELVEPLTPSGAMPNQAQMRILKETELRKVKVLGSGAFGTVYKGIWIPDGENVKIPVAIKVLRENTSPKANKEILDEAYVMAGVGSPYVSRLLGICLTSTVQLVTQLMPYGCLLDHVRENRGRLGSQDLLNWCMQIAKGMSYLEDVRLVHRDLAARNVLVKSPNHVKITDFGLARLLDIDETEYHADGGKVPIKWMALESILRRRFTHQSDVWSYGVTVWELMTFGAKPYDGIPAREIPDLLEKGERLPQPPICTIDVYMIMVKCWMIDSECRPRFRELVSEFSRMARDPQRFVVIQNEDLGPASPLDSTFYRSLLEDDDMGDLVDAEEYLVPQQGFFCPDPAPGAGGMVHHRHRSSSTRSGGGDLTLGLEPSEEEAPRSPLAPSEGAGSDVFDGDLGMGAAKGLQSLPTHDPSPLQRYSEDPTVPLPSETDGYVAPLTCSPQPEYVNQPDVRPQPPSPREGPLPAARPAGATLERPKTLSPGKNGVVKDVFAFGGAVENPEYLTPQGGAAPQPHPPPAFSPAFDNLYYWDQDPPERGAPPSTFKGTPTAENPEYLGLDVPV",
# GNAI2_HUMAN
"MGCTVSAEDKAAAERSKMIDKNLREDGEKAAREVKLLLLGAGESGKSTIVKQMKIIHEDGYSEEECRQYRAVVYSNTIQSIMAIVKAMGNLQIDFADPSRADDARQLFALSCTAEEQGVLPDDLSGVIRRLWADHGVQACFGRSREYQLNDSAAYYLNDLERIAQSDYIPTQQDVLRTRVKTTGIVETHFTFKDLHFKMFDVGGQRSERKKWIHCFEGVTAIIFCVALSAYDLVLAEDEEMNRMHESMKLFDSICNNKWFTDTSIILFLNKKDLFEEKITHSPLTICFPEYTGANKYDEAASYIQSKFEDLNKRKDTKEIYTHFTCATDTKNVQFVFDAVTDVIIKNNLKDCGLF",
# IGF1_HUMAN
"MGKISSLPTQLFKCCFCDFLKVKMHTMSSSHLFYLALCLLTFTSSATAGPETLCGAELVDALQFVCGDRGFYFNKPTGYGSSSRRAPQTGIVDECCFRSCDLRRLEMYCAPLKPAKSARSVRAQRHTDMPKTQKYQPPSTNKNTKSQRRKGWPKTHPGGEQKEGTEASLQIRGKKKEQRREIGSRNAECRGKKGK",
# A4_HUMAN
"MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMNVQNGKWDSDPSGTKTCIDTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRGRKQCKTHPHFVIPYRCLVGEFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFRGVEFVCCPLAEESDNVDSADAEEDDSDVWWGGADTDYADGSEDKVVEVAEEEEVAEVEEEEADDDEDDEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSAMSQSLLKTTQEPLARDPVKLPTTAASTPDAVDKYLETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQAKNLPKADKKAVIQHFQEKVESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITALQAVPPRPRHVFNMLKKYVRAEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYERMNQSLSLLYNVPAVAEEIQDEVDELLQKEQNYSDDVLANMISEPRISYGNDALMPSLTETKTTVELLPVNGEFSLDDLQPWHSFGADSVPANTENEVEPVDARPAADRGLTTRPGSGLTNIKTEEISEVKMDAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIATVIVITLVMLKKKQYTSIHHGVVEVDAAVTPEERHLSKMQQNGYENPTYKFFEQMQN",
# ITB3_HUMAN
"MRARPRPRPLWATVLALGALAGVGVGGPNICTTRGVSSCQQCLAVSPMCAWCSDEALPLGSPRCDLKENLLKDNCAPESIEFPVSEARVLEDRPLSDKGSGDSSQVTQVSPQRIALRLRPDDSKNFSIQVRQVEDYPVDIYYLMDLSYSMKDDLWSIQNLGTKLATQMRKLTSNLRIGFGAFVDKPVSPYMYISPPEALENPCYDMKTTCLPMFGYKHVLTLTDQVTRFNEEVKKQSVSRNRDAPEGGFDAIMQATVCDEKIGWRNDASHLLVFTTDAKTHIALDGRLAGIVQPNDGQCHVGSDNHYSASTTMDYPSLGLMTEKLSQKNINLIFAVTENVVNLYQNYSELIPGTTVGVLSMDSSNVLQLIVDAYGKIRSKVELEVRDLPEELSLSFNATCLNNEVIPGLKSCMGLKIGDTVSFSIEAKVRGCPQEKEKSFTIKPVGFKDSLIVQVTFDCDCACQAQAEPNSHRCNNGNGTFECGVCRCGPGWLGSQCECSEEDYRPSQQDECSPREGQPVCSQRGECLCGQCVCHSSDFGKITGKYCECDDFSCVRYKGEMCSGHGQCSCGDCLCDSDWTGYYCNCTTRTDTCMSSNGLLCSGRGKCECGSCVCIQPGSYGDTCEKCPTCPDACTFKKECVECKKFDRGALHDENTCNRYCRDEIESVKELKDTGKDAVNCTYKNEDDCVVRFQYYEDSSGKSILYVVEEPECPKGPDILVVLLSVMGAILLIGLAALLIWKLLITIHDRKEFAKFEEERARAKWDTANNPLYKEATSTFTNITYRGT",
# KPCG_HUMAN
"MAGLGPGVGDSEGGPRPLFCRKGALRQKVVHEVKSHKFTARFFKQPTFCSHCTDFIWGIGKQGLQCQVCSFVVHRRCHEFVTFECPGAGKGPQTDDPRNKHKFRLHSYSSPTFCDHCGSLLYGLVHQGMKCSCCEMNVHRRCVRSVPSLCGVDHTERRGRLQLEIRAPTADEIHVTVGEARNLIPMDPNGLSDPYVKLKLIPDPRNLTKQKTRTVKATLNPVWNETFVFNLKPGDVERRLSVEVWDWDRTSRNDFMGAMSFGVSELLKAPVDGWYKLLNQEEGEYYNVPVADADNCSLLQKFEACNYPLELYERVRMGPSSSPIPSPSPSPTDPKRCFFGASPGRLHISDFSFLMVLGKGSFGKVMLAERRGSDELYAIKILKKDVIVQDDDVDCTLVEKRVLALGGRGPGGRPHFLTQLHSTFQTPDRLYFVMEYVTGGDLMYHIQQLGKFKEPHAAFYAAEIAIGLFFLHNQGIIYRDLKLDNVMLDAEGHIKITDFGMCKENVFPGTTTRTFCGTPDYIAPEIIAYQPYGKSVDWWSFGVLLYEMLAGQPPFDGEDEEELFQAIMEQTVTYPKSLSREAVAICKGFLTKHPGKRLGSGPDGEPTIRAHGFFRWIDWERLERLEIPPPFRPRPCGRSGENFDKFFTRAAPALTPPDRLVLASIDQADFQGFTYVNPDFVHPDARSPTSPVPVPVM",
# IF2A_HUMAN
"MPGLSCRFYQHKFPEVEDVVMVNVRSIAEMGAYVSLLEYNNIEGMILLSELSRRRIRSINKLIRIGRNECVVVIRVDKEKGYIDLSKRRVSPEEAIKCEDKFTKSKTVYSILRHVAEVLEYTKDEQLESLFQRTAWVFDDKYKRPGYGAYDAFKHAVSDPSILDSLDLNEDEREVLINNINRRLTPQAVKIRADIEVACYGYEGIDAVKEALRAGLNCSTENMPIKINLIAPPRYVMTTTTLERTEGLSVLSQAMAVIKEKIEEKRGVFNVQMEPKVVTDTDETELARQMERLERENAEVDGDDDAEEMEAKAED",
# RLA1_HUMAN
"MASVSELACIYSALILHDDEVTVTEDKINALIKAAGVNVEPFWPGLFAKALANVNIGSLICNVGAGGPAPAAGAAPAGGPAPSTAAAPAEEKKVEAKKEESEESDDDMGFGLFD",
# RLA2_HUMAN
"MRYVASYLLAALGGNSSPSAKDIKKILDSVGIEADDDRLNKVISELNGKNIEDVIAQGIGKLASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEKKDEKKEESEESDDDMGFGLFD",
# RLA0_HUMAN
"MPREDRATWKSNYFLKIIQLLDDYPKCFIVGADNVGSKQMQQIRMSLRGKAVVLMGKNTMMRKAIRGHLENNPALEKLLPHIRGNVGFVFTKEDLTEIRDMLLANKVPAAARAGAIAPCEVTVPAQNTGLGPEKTSFFQALGITTKISRGTIEILSDVQLIKTGDKVGASEATLLNMLNISPFSFGLVIQQVFDNGSIYNPEVLDITEETLHSRFLEGVRNVASVCLQIGYPTVASVPHSIINGYKRVLALSVETDYTFPLAEKVKAFLADPSAFVAAAPVAAATTAAPAAAAAPAKVEAKEESEESDEDMGFGLFD",
# ITB1_HUMAN
"MNLQPIFWIGLISSVCCVFAQTDENRCLKANAKSCGECIQAGPNCGWCTNSTFLQEGMPTSARCDDLEALKKKGCPPDDIENPRGSKDIKKNKNVTNRSKGTAEKLKPEDITQIQPQQLVLRLRSGEPQTFTLKFKRAEDYPIDLYYLMDLSYSMKDDLENVKSLGTDLMNEMRRITSDFRIGFGSFVEKTVMPYISTTPAKLRNPCTSEQNCTSPFSYKNVLSLTNKGEVFNELVGKQRISGNLDSPEGGFDAIMQVAVCGSLIGWRNVTRLLVFSTDAGFHFAGDGKLGGIVLPNDGQCHLENNMYTMSHYYDYPSIAHLVQKLSENNIQTIFAVTEEFQPVYKELKNLIPKSAVGTLSANSSNVIQLIIDAYNSLSSEVILENGKLSEGVTISYKSYCKNGVNGTGENGRKCSNISIGDEVQFEISITSNKCPKKDSDSFKIRPLGFTEEVEVILQYICECECQSEGIPESPKCHEGNGTFECGACRCNEGRVGRHCECSTDEVNSEDMDAYCRKENSSEICSNNGECVCGQCVCRKRDNTNEIYSGKFCECDNFNCDRSNGLICGGNGVCKCRVCECNPNYTGSACDCSLDTSTCEASNGQICNGRGICECGVCKCTDPKFQGQTCEMCQTCLGVCAEHKECVQCRAFNKGEKKDTCTQECSYFNITKVESRDKLPQPVQPDPVSHCKEKDVDDCWFYFTYSVNGNNEVMVHVVENPECPTGPDIIPIVAGVVAGIVLIGLALLLIWKLLMIIHDRREFAKFEKEKMNAKWDTGENPIYKSAVTTVVNPKYEGK",
# KPCB_HUMAN
"MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQPTFCSHCTDFIWGFGKQGFQCQVCCFVVHKRCHEFVTFSCPGADKGPASDDPRSKHKFKIHTYSSPTFCDHCGSLLYGLIHQGMKCDTCMMNVHKRCVMNVPSLCGTDHTERRGRIYIQAHIDRDVLIVLVRDAKNLVPMDPNGLSDPYVKLKLIPDPKSESKQKTKTIKCSLNPEWNETFRFQLKESDKDRRLSVEIWDWDLTSRNDFMGSLSFGISELQKASVDGWFKLLSQEEGEYFNVPVPPEGSEANEELRQKFERAKISQGTKVPEEKTTNTVSKFDNNGNRDRMKLTDFNFLMVLGKGSFGKVMLSERKGTDELYAVKILKKDVVIQDDDVECTMVEKRVLALPGKPPFLTQLHSCFQTMDRLYFVMEYVNGGDLMYHIQQVGRFKEPHAVFYAAEIAIGLFFLQSKGIIYRDLKLDNVMLDSEGHIKIADFGMCKENIWDGVTTKTFCGTPDYIAPEIIAYQPYGKSVDWWAFGVLLYEMLAGQAPFEGEDEDELFQSIMEHNVAYPKSMSKEAVAICKGLMTKHPGKRLGCGPEGERDIKEHAFFRYIDWEKLERKEIQPPYKPKARDKRDTSNFDKEFTRQPVELTPTDKLFIMNLDQNEFAGFSYTNPEFVINV",
# CALB1_HUMAN
"MAESHLQSSLITASQFFEIWLHFDADGSGYLEGKELQNLIQELQQARKKAGLELSPEMKTFVDQYGQRDDGKIGIVELAHVLPTEENFLLLFRCQQLKSCEEFMKTWRKYDTDHSGFIETEELKNFLKDLLEKANKTVDDTKLAEYTDLMLKLFDSNNDGKLELTEMARLLPVQENFLLKFQGIKMCGKEFNKAFELYDQDGNGYIDENELDALLKDLCEKNKQDLDINNITTYKKNIMALSDGGKLYRTDLALILCAGDN",
# LCK_HUMAN
"MGCGCSSHPEDDWMENIDVCENCHYPIVPLDGKGTLLIRNGSEVRDPLVTYEGSNPPASPLQDNLVIALHSYEPSHDGDLGFEKGEQLRILEQSGEWWKAQSLTTGQEGFIPFNFVAKANSLEPEPWFFKNLSRKDAERQLLAPGNTHGSFLIRESESTAGSFSLSVRDFDQNQGEVVKHYKIRNLDNGGFYISPRITFPGLHELVRHYTNASDGLCTRLSRPCQTQKPQKPWWEDEWEVPRETLKLVERLGAGQFGEVWMGYYNGHTKVAVKSLKQGSMSPDAFLAEANLMKQLQHQRLVRLYAVVTQEPIYIITEYMENGSLVDFLKTPSGIKLTINKLLDMAAQIAEGMAFIEERNYIHRDLRAANILVSDTLSCKIADFGLARLIEDNEYTAREGAKFPIKWTAPEAINYGTFTIKSDVWSFGILLTEIVTHGRIPYPGMTNPEVIQNLERGYRMVRPDNCPEELYQLMRLCWKERPEDRPTFDYLRSVLEDFFTATEGQYQPQP",
# FYN_HUMAN
"MGCVQCKDKEATKLTEERDGSLNQSSGYRYGTDPTPQHYPSFGVTSIPNYNNFHAAGGQGLTVFGGVNSSSHTGTLRTRGGTGVTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSLTTGETGYIPSNYVAPVDSIQAEEWYFGKLGRKDAERQLLSFGNPRGTFLIRESETTKGAYSLSIRDWDDMKGDHVKHYKIRKLDNGGYYITTRAQFETLQQLVQHYSERAAGLCCRLVVPCHKGMPRLTDLSVKTKDVWEIPRESLQLIKRLGNGQFGEVWMGTWNGNTKVAIKTLKPGTMSPESFLEEAQIMKKLKHDKLVQLYAVVSEEPIYIVTEYMNKGSLLDFLKDGEGRALKLPNLVDMAAQVAAGMAYIERMNYIHRDLRSANILVGNGLICKIADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELVTKGRVPYPGMNNREVLEQVERGYRMPCPQDCPISLHELMIHCWKKDPEERPTFEYLQSFLEDYFTATEPQYQPGENL",
# CCKN_HUMAN
"MNSGVCLCVLMAVLAAGALTQPVPPADPAGSGLQRAEEAPRRQLRVSQRTDGESRAHLGALLARYIQQARKAPSGRMSIVKNLQNLDPSHRISDRDYMGWMDFGRRSAEEYEYPS",
# APOA4_HUMAN
"MFLKAVVLTLALVAVAGARAEVSADQVATVMWDYFSQLSNNAKEAVEHLQKSELTQQLNALFQDKLGEVNTYAGDLQKKLVPFATELHERLAKDSEKLKEEIGKELEELRARLLPHANEVSQKIGDNLRELQQRLEPYADQLRTQVSTQAEQLRRQLTPYAQRMERVLRENADSLQASLRPHADELKAKIDQNVEELKGRLTPYADEFKVKIDQTVEELRRSLAPYAQDTQEKLNHQLEGLTFQMKKNAEELKARISASAEELRQRLAPLAEDVRGNLRGNTEGLQKSLAELGGHLDQQVEEFRRRVEPYGENFNKALVQQMEQLRQKLGPHAGDVEGHLSFLEKDLRDKVNSFFSTFKEKESQDKTLSLPELEQQQEQQQEQQQEQVQMLAPLES",
# IF4E_HUMAN
"MATVEPETTPTPNPPTTEEEKTESNQEVANPEHYIKHPLQNRWALWFFKNDKSKTWQANLRLISKFDTVEDFWALYNHIQLSSNLMPGCDYSLFKDGIEPMWEDEKNKRGGRWLITLNKQQRRSDLDRFWLETLLCLIGESFDDYSDDVCGAVVNVRAKGDKIAIWTTECENREAVTHIGRVYKERLGLPPKIVIGYQSHADTATKSGSTTKNRFVV",
# CRF_HUMAN
"MRLPLLVSAGVLLVALLPCPPCRALLSRGPVPGARQAPQHPQPLDFFQPPPQSEQPQQPQARPVLLRMGEEYFLRLGNLNKSPAAPLSPASSLLAGGSGSRPSPEQATANFFRVLLQQLLLPRRSLDSPAALAERGARNALGGHQEAPERERRSEEPPISLDLTFHLLREVLEMARAEQLAQQAHSNRKLMEIIGK",
# CALCA_HUMAN
"MGFQKFSPFLALSILVLLQAGSLHAAPFRSALESSPADPATLSEDEARLLLAALVQDYVQMKASELEQEQEREGSRIIAQKRACDTATCVTHRLAGLLSRSGGVVKNNFVPTNVGSKAFGRRRRDLQA",
# NFL_HUMAN
"MSSFSYEPYYSTSYKRRYVETPRVHISSVRSGYSTARSAYSSYSAPVSSSLSVRRSYSSSSGSLMPSLENLDLSQVAAISNDLKSIRTQEKAQLQDLNDRFASFIERVHELEQQNKVLEAELLVLRQKHSEPSRFRALYEQEIRDLRLAAEDATNEKQALQGEREGLEETLRNLQARYEEEVLSREDAEGRLMEARKGADEAALARAELEKRIDSLMDEISFLKKVHEEEIAELQAQIQYAQISVEMDVTKPDLSAALKDIRAQYEKLAAKNMQNAEEWFKSRFTVLTESAAKNTDAVRAAKDEVSESRRLLKAKTLEIEACRGMNEALEKQLQELEDKQNADISAMQDTINKLENELRTTKSEMARYLKEYQDLLNVKMALDIEIAAYRKLLEGEETRLSFTSVGSITSGYSQSSQVFGRSAYGGLQTSSYLMSTRSFPSYYTSHVQEEQIEVEETIEAAKAEEAKDEPPSEGEAEEEEKDKEEAEEEEAAEEEEAAKEESEEAKEEEEGGEGEEGEETKEAEEEEKKVEGAGEEQAAKKKD",
# NFM_HUMAN
"MSYTLDSLGNPSAYRRVTETRSSFSRVSGSPSSGFRSQSWSRGSPSTVSSSYKRSMLAPRLAYSSAMLSSAESSLDFSQSSSLLNGGSGPGGDYKLSRSNEKEQLQGLNDRFAGYIEKVHYLEQQNKEIEAEIQALRQKQASHAQLGDAYDQEIRELRATLEMVNHEKAQVQLDSDHLEEDIHRLKERFEEEARLRDDTEAAIRALRKDIEEASLVKVELDKKVQSLQDEVAFLRSNHEEEVADLLAQIQASHITVERKDYLKTDISTALKEIRSQLESHSDQNMHQAEEWFKCRYAKLTEAAEQNKEAIRSAKEEIAEYRRQLQSKSIELESVRGTKESLERQLSDIEERHNHDLSSYQDTIQQLENELRGTKWEMARHLREYQDLLNVKMALDIEIAAYRKLLEGEETRFSTFAGSITGPLYTHRPPITISSKIQKPKVEAPKLKVQHKFVEEIIEETKVEDEKSEMEEALTAITEELAVSMKEEKKEAAEEKEEEPEAEEEEVAAKKSPVKATAPEVKEEEGEKEEEEGQEEEEEEDEGAKSDQAEEGGSEKEGSSEKEEGEQEEGETEAEAEGEEAEAKEEKKVEEKSEEVATKEELVADAKVEKPEKAKSPVPKSPVEEKGKSPVPKSPVEEKGKSPVPKSPVEEKGKSPVPKSPVEEKGKSPVSKSPVEEKAKSPVPKSPVEEAKSKAEVGKGEQKEEEEKEVKEAPKEEKVEKKEEKPKDVPEKKKAESPVKEEAVAEVVTITKSVKVHLEKETKEEGKPLQQEKEKEKAGGEGGSEEEGSDKGAKGSRKEDIAVNGEVEGKEEVEQETKEKGSGREEEKGVVTNGLDLSPADEKKGGDKSEEKVVVTKTVEKITSEGGDGATKYITKSVTVTQKVEEHEETFEEKLVSTKKVEKVTSHAIVKEVTQSD",
# CATD_HUMAN
"MQPSSLLPLALCLLAAPASALVRIPLHKFTSIRRTMSEVGGSVEDLIAKGPVSKYSQAVPAVTEGPIPEVLKNYMDAQYYGEIGIGTPPQCFTVVFDTGSSNLWVPSIHCKLLDIACWIHHKYNSDKSSTYVKNGTSFDIHYGSGSLSGYLSQDTVSVPCQSASSASALGGVKVERQVFGEATKQPGITFIAAKFDGILGMAYPRISVNNVLPVFDNLMQQKLVDQNIFSFYLSRDPDAQPGGELMLGGTDSKYYKGSLSYLNVTRKAYWQVHLDQVEVASGLTLCKEGCEAIVDTGTSLMVGPVDEVRELQKAIGAVPLIQGEYMIPCEKVSTLPAITLKLGGKGYKLSPEDYTLKVSQAGKTLCLSGFMGMDIPPPSGPLWILGDVFIGRYYTVFDRDNNRVGFAEAARL",
# CAN1_HUMAN
"MSEEIITPVYCTGVSAQVQKQRARELGLGRHENAIKYLGQDYEQLRVRCLQSGTLFRDEAFPPVPQSLGYKDLGPNSSKTYGIKWKRPTELLSNPQFIVDGATRTDICQGALGDCWLLAAIASLTLNDTLLHRVVPHGQSFQNGYAGIFHFQLWQFGEWVDVVVDDLLPIKDGKLVFVHSAEGNEFWSALLEKAYAKVNGSYEALSGGSTSEGFEDFTGGVTEWYELRKAPSDLYQIILKALERGSLLGCSIDISSVLDMEAITFKKLVKGHAYSVTGAKQVNYRGQVVSLIRMRNPWGEVEWTGAWSDSSSEWNNVDPYERDQLRVKMEDGEFWMSFRDFMREFTRLEICNLTPDALKSRTIRKWNTTLYEGTWRRGSTAGGCRNYPATFWVNPQFKIRLDETDDPDDYGDRESGCSFVLALMQKHRRRERRFGRDMETIGFAVYEVPPELVGQPAVHLKRDFFLANASRARSEQFINLREVSTRFRLPPGEYVVVPSTFEPNKEGDFVLRFFSEKSAGTVELDDQIQANLPDEQVLSEEEIDENFKALFRQLAGEDMEISVKELRTILNRIISKHKDLRTKGFSLESCRSMVNLMDRDGNGKLGLVEFNILWNRIRNYLSIFRKFDLDKSGSMSAYEMRMAIESAGFKLNKKLYELIITRYSEPDLAVDFDNFVCCLVRLETMFRFFKTLDTDLDGVVTFDLFKWLQLTMFA",
# TBB5_HUMAN
"MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGTYHGDSDLQLDRISVYYNEATGGKYVPRAILVDLEPGTMDSVRSGPFGQIFRPDNFVFGQSGAGNNWAKGHYTEGAELVDSVLDVVRKEAESCDCLQGFQLTHSLGGGTGSGMGTLLISKIREEYPDRIMNTFSVVPSPKVSDTVVEPYNATLSVHQLVENTDETYCIDNEALYDICFRTLKLTTPTYGDLNHLVSATMSGVTTCLRFPGQLNADLRKLAVNMVPFPRLHFFMPGFAPLTSRGSQQYRALTVPELTQQVFDAKNMMAACDPRHGRYLTVAAVFRGRMSMKEVDEQMLNVQNKNSSYFVEWIPNNVKTAVCDIPPRGLKMAVTFIGNSTAIQELFKRISEQFTAMFRRKAFLHWYTGEGMDEMEFTEAESNMNDLVSEYQQYQDATAEEEEDFGEEAEEEA",
# GRP_HUMAN
"MRGRELPLVLLALVLCLAPRGRAVPLPAGGGTVLTKMYPRGNHWAVGHLMGKKSTGESSSVSERGSLKQQLREYIRWEEAARNLLGLIEAKENRNHQPPQPKALGNQQPSWDSEDSSNFKDVGSKGKVGRLSAPGSQREGRNPQLNQQ",
# CASK_HUMAN
"MKSFLLVVNALALTLPFLAVEVQNQKQPACHENDERPFYQKTAPYVPMYYVPNSYPYYGTNLYQRRPAIAINNPYVPRTYYANPAVVRPHAQIPQRQYLPNSHPPTVVRRPNLHPSFIAIPPKKIQDKIIIPTINTIATVEPTPAPATEPTVDSVVTPEAFSESIITSTPETTTVAVTPPTA",
# ACHG_HUMAN
"MHGGQGPLLLLLLLAVCLGAQGRNQEERLLADLMQNYDPNLRPAERDSDVVNVSLKLTLTNLISLNEREEALTTNVWIEMQWCDYRLRWDPRDYEGLWVLRVPSTMVWRPDIVLENNVDGVFEVALYCNVLVSPDGCIYWLPPAIFRSACSISVTYFPFDWQNCSLIFQSQTYSTNEIDLQLSQEDGQTIEWIFIDPEAFTENGEWAIQHRPAKMLLDPAAPAQEAGHQKVVFYLLIQRKPLFYVINIIAPCVLISSVAILIHFLPAKAGGQKCTVAINVLLAQTVFLFLVAKKVPETSQAVPLISKYLTFLLVVTILIVVNAVVVLNVSLRSPHTHSMARGVRKVFLRLLPQLLRMHVRPLAPAAVQDTQSRLQNGSSGWSITTGEEVALCLPRSELLFQQWQRQGLVAAALEKLEKGPELGLSQFCGSLKQAAPAIQACVEACNLIACARHQQSHFDNGNEEWFLVGRVLDRVCFLAMLSLFICGTAGIFLMAHYNRVPALPFPGDPRPYLPSPD",
# ADRB2_HUMAN
"MGQPGNGSAFLLAPNGSHAPDHDVTQERDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSLACADLVMGLAVVPFGAAHILMKMWTFGNFWCEFWTSIDVLCVTASIETLCVIAVDRYFAITSPFKYQSLLTKNKARVIILMVWIVSGLTSFLPIQMHWYRATHQEAINCYANETCCDFFTNQAYAIASSIVSFYVPLVIMVFVYSRVFQEAKRQLQKIDKSEGRFHVQNLSQVEQDGRTGHGLRRSSKFCLKEHKALKTLGIIMGTFTLCWLPFFIVNIVHVIQDNLIRKEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCLRRSSLKAYGNGYSSNGNTGEQSGYHVEQEKENKLLCEDLPGTEDFVGHQGTVPSDNIDSQGRNCSTNDSLL",
# PROF1_HUMAN
"MAGWNAYIDNLMADGTCQDAAIVGYKDSPSVWAAVPGKTFVNITPAEVGVLVGKDRSSFYVNGLTLGGQKCSVIRDSLLQDGEFSMDLRTKSTGGAPTFNVTVTKTDKTLVLLMGKEGVHGGLINKKCYEMASHLRRSQY",
# HS90A_HUMAN
"MPEETQTQDQPMEEEEVETFAFQAEIAQLMSLIINTFYSNKEIFLRELISNSSDALDKIRYESLTDPSKLDSGKELHINLIPNKQDRTLTIVDTGIGMTKADLINNLGTIAKSGTKAFMEALQAGADISMIGQFGVGFYSAYLVAEKVTVITKHNDDEQYAWESSAGGSFTVRTDTGEPMGRGTKVILHLKEDQTEYLEERRIKEIVKKHSQFIGYPITLFVEKERDKEVSDDEAEEKEDKEEEKEKEEKESEDKPEIEDVGSDEEEEKKDGDKKKKKKIKEKYIDQEELNKTKPIWTRNPDDITNEEYGEFYKSLTNDWEDHLAVKHFSVEGQLEFRALLFVPRRAPFDLFENRKKKNNIKLYVRRVFIMDNCEELIPEYLNFIRGVVDSEDLPLNISREMLQQSKILKVIRKNLVKKCLELFTELAEDKENYKKFYEQFSKNIKLGIHEDSQNRKKLSELLRYYTSASGDEMVSLKDYCTRMKENQKHIYYITGETKDQVANSAFVERLRKHGLEVIYMIEPIDEYCVQQLKEFEGKTLVSVTKEGLELPEDEEEKKKQEEKKTKFENLCKIMKDILEKKVEKVVVSNRLVTSPCCIVTSTYGWTANMERIMKAQALRDNSTMGYMAAKKHLEINPDHSIIETLRQKAEADKNDKSVKDLVILLYETALLSSGFSLEDPQTHANRIYRMIKLGLGIDEDDPTADDTSAAVTEEMPPLEGDDDTSRMEEVD",
# YES_HUMAN
"MGCIKSKENKSPAIKYRPENTPEPVSTSVSHYGAEPTTVSPCPSSSAKGTAVNFSSLSMTPFGGSSGVTPFGGASSSFSVVPSSYPAGLTGGVTIFVALYDYEARTTEDLSFKKGERFQIINNTEGDWWEARSIATGKNGYIPSNYVAPADSIQAEEWYFGKMGRKDAERLLLNPGNQRGIFLVRESETTKGAYSLSIRDWDEIRGDNVKHYKIRKLDNGGYYITTRAQFDTLQKLVKHYTEHADGLCHKLTTVCPTVKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTKVAIKTLKPGTMMPEAFLQEAQIMKKLRHDKLVPLYAVVSEEPIYIVTEFMSKGSLLDFLKEGDGKYLKLPQLVDMAAQIADGMAYIERMNYIHRDLRAANILVGENLVCKIADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILQTELVTKGRVPYPGMVNREVLEQVERGYRMPCPQGCPESLHELMNLCWKKDPDERPTFEYIQSFLEDYFTATEPQYQPGENL",
# LYN_HUMAN
"MGCIKSKGKDSLSDDGVDLKTQPVRNTERTIYVRDPTSNKQQRPVPESQLLPGQRFQTKDPEEQGDIVVALYPYDGIHPDDLSFKKGEKMKVLEEHGEWWKAKSLLTKKEGFIPSNYVAKLNTLETEEWFFKDITRKDAERQLLAPGNSAGAFLIRESETLKGSFSLSVRDFDPVHGDVIKHYKIRSLDNGGYYISPRITFPCISDMIKHYQKQADGLCRRLEKACISPKPQKPWDKDAWEIPRESIKLVKRLGAGQFGEVWMGYYNNSTKVAVKTLKPGTMSVQAFLEEANLMKTLQHDKLVRLYAVVTREEPIYIITEYMAKGSLLDFLKSDEGGKVLLPKLIDFSAQIAEGMAYIERKNYIHRDLRAANVLVSESLMCKIADFGLARVIEDNEYTAREGAKFPIKWTAPEAINFGCFTIKSDVWSFGILLYEIVTYGKIPYPGRTNADVMTALSQGYRMPRVENCPDELYDIMKMCWKEKAEERPTFDYLQSVLDDFYTATEGQYQQQP",
# IGF1R_HUMAN
"MKSGSGGGSPTSLWGLLFLSAALSLWPTSGEICGPGIDIRNDYQQLKRLENCTVIEGYLHILLISKAEDYRSYRFPKLTVITEYLLLFRVAGLESLGDLFPNLTVIRGWKLFYNYALVIFEMTNLKDIGLYNLRNITRGAIRIEKNADLCYLSTVDWSLILDAVSNNYIVGNKPPKECGDLCPGTMEEKPMCEKTTINNEYNYRCWTTNRCQKMCPSTCGKRACTENNECCHPECLGSCSAPDNDTACVACRHYYYAGVCVPACPPNTYRFEGWRCVDRDFCANILSAESSDSEGFVIHDGECMQECPSGFIRNGSQSMYCIPCEGPCPKVCEEEKKTKTIDSVTSAQMLQGCTIFKGNLLINIRRGNNIASELENFMGLIEVVTGYVKIRHSHALVSLSFLKNLRLILGEEQLEGNYSFYVLDNQNLQQLWDWDHRNLTIKAGKMYFAFNPKLCVSEIYRMEEVTGTKGRQSKGDINTRNNGERASCESDVLHFTSTTTSKNRIIITWHRYRPPDYRDLISFTVYYKEAPFKNVTEYDGQDACGSNSWNMVDVDLPPNKDVEPGILLHGLKPWTQYAVYVKAVTLTMVENDHIRGAKSEILYIRTNASVPSIPLDVLSASNSSSQLIVKWNPPSLPNGNLSYYIVRWQRQPQDGYLYRHNYCSKDKIPIRKYADGTIDIEEVTENPKTEVCGGEKGPCCACPKTEAEKQAEKEEAEYRKVFENFLHNSIFVPRPERKRRDVMQVANTTMSSRSRNTTAADTYNITDPEELETEYPFFESRVDNKERTVISNLRPFTLYRIDIHSCNHEAEKLGCSASNFVFARTMPAEGADDIPGPVTWEPRPENSIFLKWPEPENPNGLILMYEIKYGSQVEDQRECVSRQEYRKYGGAKLNRLNPGNYTARIQATSLSGNGSWTDPVFFYVQAKTGYENFIHLIIALPVAVLLIVGGLVIMLYVFHRKRNNSRLGNGVLYASVNPEYFSAADVYVPDEWEVAREKITMSRELGQGSFGMVYEGVAKGVVKDEPETRVAIKTVNEAASMRERIEFLNEASVMKEFNCHHVVRLLGVVSQGQPTLVIMELMTRGDLKSYLRSLRPEMENNPVLAPPSLSKMIQMAGEIADGMAYLNANKFVHRDLAARNCMVAEDFTVKIGDFGMTRDIYETDYYRKGGKGLLPVRWMSPESLKDGVFTTYSDVWSFGVVLWEIATLAEQPYQGLSNEQVLRFVMEGGLLDKPDNCPDMLFELMRMCWQYNPKMRPSFLEIISSIKEEMEPGFREVSFYYSEENKLPEPEELDLEPENMESVPLDPSASSSSLPLPDRHSGHKAENGPGPGVLVLRASFDERQPYAHMNGGRKNERALPLPQSSTC",
# TNR16_HUMAN
"MGAGATGRAMDGPRLLLLLLLGVSLGGAKEACPTGLYTHSGECCKACNLGEGVAQPCGANQTVCEPCLDSVTFSDVVSATEPCKPCTECVGLQSMSAPCVEADDAVCRCAYGYYQDETTGRCEACRVCEAGSGLVFSCQDKQNTVCEECPDGTYSDEANHVDPCLPCTVCEDTERQLRECTRWADAECEEIPGRWITRSTPPEGSDSTAPSTQEPEAPPEQDLIASTVAGVVTTVMGSSQPVVTRGTTDNLIPVYCSILAAVVVGLVAYIAFKRWNSCKQNKQGANSRPVNQTPPPEGEKLHSDSGISVDSQSLHDQQPHTQTASGQALKGDGGLYSSLPPAKREEVEKLLNGSAGDTWRHLAGELGYQPEHIDSFTHEACPVRALLASWATQDSATLDALLAALRRIQRADLVESLCSESTATSPV",
# ACM2_HUMAN
"MNNSTNSSNNSLALTSPYKTFEVVFIVLVAGSLSLVTIIGNILVMVSIKVNRHLQTVNNYFLFSLACADLIIGVFSMNLYTLYTVIGYWPLGPVVCDLWLALDYVVSNASVMNLLIISFDRYFCVTKPLTYPVKRTTKMAGMMIAAAWVLSFILWAPAILFWQFIVGVRTVEDGECYIQFFSNAAVTFGTAIAAFYLPVIIMTVLYWHISRASKSRIKKDKKEPVANQDPVSPSLVQGRIVKPNNNNMPSSDDGLEHNKIQNGKAPRDPVTENCVQGEEKESSNDSTSVSAVASNMRDDEITQDENTVSTSLGHSKDENSKQTCIRIGTKTPKSDSCTPTNTTVEVVGSSGQNGDEKQNIVARKIVKMTKQPAKKKPPPSREKKVTRTILAILLAFIITWAPYNVMVLINTFCAPCIPNTVWTIGYWLCYINSTINPACYALCNATFKKTFKHLLMCHYKNIGATR",
# ACM4_HUMAN
"MANFTPVNGSSGNQSVRLVTSSSHNRYETVEMVFIATVTGSLSLVTVVGNILVMLSIKVNRQLQTVNNYFLFSLACADLIIGAFSMNLYTVYIIKGYWPLGAVVCDLWLALDYVVSNASVMNLLIISFDRYFCVTKPLTYPARRTTKMAGLMIAAAWVLSFVLWAPAILFWQFVVGKRTVPDNQCFIQFLSNPAVTFGTAIAAFYLPVVIMTVLYIHISLASRSRVHKHRPEGPKEKKAKTLAFLKSPLMKQSVKKPPPGEAAREELRNGKLEEAPPPALPPPPRPVADKDTSNESSSGSATQNTKERPATELSTTEATTPAMPAPPLQPRALNPASRWSKIQIVTKQTGNECVTAIEIVPATPAGMRPAANVARKFASIARNQVRKKRQMAARERKVTRTIFAILLAFILTWTPYNVMVLVNTFCQSCIPDTVWSIGYWLCYVNSTINPACYALCNATFKKTFRHLLLCQYRNIGTAR",
# 4F2_HUMAN
"MELQPPEASIAVVSIPRQLPGSHSEAGVQGLSAGDDSELGSHCVAQTGLELLASGDPLPSASQNAEMIETGSDCVTQAGLQLLASSDPPALASKNAEVTGTMSQDTEVDMKEVELNELEPEKQPMNAASGAAMSLAGAEKNGLVKIKVAEDEAEAAAAAKFTGLSKEELLKVAGSPGWVRTRWALLLLFWLGWLGMLAGAVVIIVRAPRCRELPAQKWWHTGALYRIGDLQAFQGHGAGNLAGLKGRLDYLSSLKVKGLVLGPIHKNQKDDVAQTDLLQIDPNFGSKEDFDSLLQSAKKKSIRVILDLTPNYRGENSWFSTQVDTVATKVKDALEFWLQAGVDGFQVRDIENLKDASSFLAEWQNITKGFSEDRLLIAGTNSSDLQQILSLLESNKDLLLTSSYLSDSGSTGEHTKSLVTQYLNATGNRWCSWSLSQARLLTSFLPAQLLRLYQLMLFTLPGTPVFSYGDEIGLDAAALPGQPMEAPVMLWDESSFPDIPGAVSANMTVKGQSEDPGSLLSLFRRLSDQRSKERSLLHGDFHAFSAGPGLFSYIRHWDQNERFLVVLNFGDVGLSAGLQASDLPASASLPAKADLLLSTQPGREEGSPLELERLKLEPHEGLLLRFPYAA",
# MCR_HUMAN
"METKGYHSLPEGLDMERRWGQVSQAVERSSLGPTERTDENNYMEIVNVSCVSGAIPNNSTQGSSKEKQELLPCLQQDNNRPGILTSDIKTELESKELSATVAESMGLYMDSVRDADYSYEQQNQQGSMSPAKIYQNVEQLVKFYKGNGHRPSTLSCVNTPLRSFMSDSGSSVNGGVMRAVVKSPIMCHEKSPSVCSPLNMTSSVCSPAGINSVSSTTASFGSFPVHSPITQGTPLTCSPNVENRGSRSHSPAHASNVGSPLSSPLSSMKSSISSPPSHCSVKSPVSSPNNVTLRSSVSSPANINNSRCSVSSPSNTNNRSTLSSPAASTVGSICSPVNNAFSYTASGTSAGSSTLRDVVPSPDTQEKGAQEVPFPKTEEVESAISNGVTGQLNIVQYIKPEPDGAFSSSCLGGNSKINSDSSFSVPIKQESTKHSCSGTSFKGNPTVNPFPFMDGSYFSFMDDKDYYSLSGILGPPVPGFDGNCEGSGFPVGIKQEPDDGSYYPEASIPSSAIVGVNSGGQSFHYRIGAQGTISLSRSARDQSFQHLSSFPPVNTLVESWKSHGDLSSRRSDGYPVLEYIPENVSSSTLRSVSTGSSRPSKICLVCGDEASGCHYGVVTCGSCKVFFKRAVEGQHNYLCAGRNDCIIDKIRRKNCPACRLQKCLQAGMNLGARKSKKLGKLKGIHEEQPQQQQPPPPPPPPQSPEEGTTYIAPAKEPSVNTALVPQLSTISRALTPSPVMVLENIEPEIVYAGYDSSKPDTAENLLSTLNRLAGKQMIQVVKWAKVLPGFKNLPLEDQITLIQYSWMCLSSFALSWRSYKHTNSQFLYFAPDLVFNEEKMHQSAMYELCQGMHQISLQFVRLQLTFEEYTIMKVLLLLSTIPKDGLKSQAAFEEMRTNYIKELRKMVTKCPNNSGQSWQRFYQLTKLLDSMHDLVSDLLEFCFYTFRESHALKVEFPAMLVEIISDQLPKVESGNAKPLYFHRK",
# SYPH_HUMAN
"MLLLADMDVVNQLVAGGQFRVVKEPLGFVKVLQWVFAIFAFATCGSYSGELQLSVDCANKTESDLSIEVEFEYPFRLHQVYFDAPTCRGGTTKVFLVGDYSSSAEFFVTVAVFAFLYSMGALATYIFLQNKYRENNKGPMLDFLATAVFAFMWLVSSSAWAKGLSDVKMATDPENIIKEMPVCRQTGNTCKELRDPVTSGLNTSVVFGFLNLVLWVGNLWFVFKETGWAAPFLRAPPGAPEKQPAPGDAYGDAGYGQGPGGYGPQDSYGPQGGYQPDYGQPAGSGGSGYGPQGDYGQQGYGPQGAPTSFSNQM",
# ADRB1_HUMAN
"MGAGVLVLGASEPGNLSSAAPLPDGAATAARLLVPASPPASLLPPASESPEPLSQQWTAGMGLLMALIVLLIVAGNVLVIVAIAKTPRLQTLTNLFIMSLASADLVMGLLVVPFGATIVVWGRWEYGSFFCELWTSVDVLCVTASIETLCVIALDRYLAITSPFRYQSLLTRARARGLVCTVWAISALVSFLPILMHWWRAESDEARRCYNDPKCCDFVTNRAYAIASSVVSFYVPLCIMAFVYLRVFREAQKQVKKIDSCERRFLGGPARPPSPSPSPVPAPAPPPGPPRPAAAAATAPLANGRAGKRRPSRLVALREQKALKTLGIIMGVFTLCWLPFFLANVVKAFHRELVPDRLFVFFNWLGYANSAFNPIIYCRSPDFRKAFQGLLCCARRAARRRHATHGDRPRASGCLARPGPPPSPGAASDDDDDDVVGATPPARLLEPWAGCNGGAAADSDSSLDEPCRPGFASESKV",
# ITA5_HUMAN
"MGSRTPESPLHAVQLRWGPRRRPPLLPLLLLLLPPPPRVGGFNLDAEAPAVLSGPPGSFFGFSVEFYRPGTDGVSVLVGAPKANTSQPGVLQGGAVYLCPWGASPTQCTPIEFDSKGSRLLESSLSSSEGEEPVEYKSLQWFGATVRAHGSSILACAPLYSWRTEKEPLSDPVGTCYLSTDNFTRILEYAPCRSDFSWAAGQGYCQGGFSAEFTKTGRVVLGGPGSYFWQGQILSATQEQIAESYYPEYLINLVQGQLQTRQASSIYDDSYLGYSVAVGEFSGDDTEDFVAGVPKGNLTYGYVTILNGSDIRSLYNFSGEQMASYFGYAVAATDVNGDGLDDLLVGAPLLMDRTPDGRPQEVGRVYVYLQHPAGIEPTPTLTLTGHDEFGRFGSSLTPLGDLDQDGYNDVAIGAPFGGETQQGVVFVFPGGPGGLGSKPSQVLQPLWAASHTPDFFGSALRGGRDLDGNGYPDLIVGSFGVDKAVVYRGRPIVSASASLTIFPAMFNPEERSCSLEGNPVACINLSFCLNASGKHVADSIGFTVELQLDWQKQKGGVRRALFLASRQATLTQTLLIQNGAREDCREMKIYLRNESEFRDKLSPIHIALNFSLDPQAPVDSHGLRPALHYQSKSRIEDKAQILLDCGEDNICVPDLQLEVFGEQNHVYLGDKNALNLTFHAQNVGEGGAYEAELRVTAPPEAEYSGLVRHPGNFSSLSCDYFAVNQSRLLVCDLGNPMKAGASLWGGLRFTVPHLRDTKKTIQFDFQILSKNLNNSQSDVVSFRLSVEAQAQVTLNGVSKPEAVLFPVSDWHPRDQPQKEEDLGPAVHHVYELINQGPSSISQGVLELSCPQALEGQQLLYVTRVTGLNCTTNHPINPKGLELDPEGSLHHQQKREAPSRSSASSGPQILKCPEAECFRLRCELGPLHQQESQSLQLHFRVWAKTFLQREHQPFSLQCEAVYKALKMPYRILPRQLPQKERQVATAVQWTKAEGSYGVPLWIIILAILFGLLLLGLLIYILYKLGFFKRSLPYGTAMEKAQLKPPATSDA",
# RS17_HUMAN
"MGRVRTKTVKKAARVIIEKYYTRLGNDFHTNKRVCEEIAIIPSKKLRNKIAGYVTHLMKRIQRGPVRGISIKLQEEERERRDNYVPEVSALDQEIIEVDPDTKEMLKLLDFGSLSNLQVTQPTVGMNFKTPRGPV",
# ANXA5_HUMAN
"MAQVLRGTVTDFPGFDERADAETLRKAMKGLGTDEESILTLLTSRSNAQRQEISAAFKTLFGRDLLDDLKSELTGKFEKLIVALMKPSRLYDAYELKHALKGAGTNEKVLTEIIASRTPEELRAIKQVYEEEYGSSLEDDVVGDTSGYYQRMLVVLLQANRDPDAGIDEAQVEQDAQALFQAGELKWGTDEEKFITIFGTRSVSHLRKVFDKYMTISGFQIEETIDRETSGNLEQLLLAVVKSIRSIPAYLAETLYYAMKGAGTDDHTLIRVMVSRSEIDLFNIRKEFRKNFATSLYSMIKGDTSGDYKKALLLLCGEDD",
# RSSA_HUMAN
"MSGALDVLQMKEEDVLKFLAAGTHLGGTNLDFQMEQYIYKRKSDGIYIINLKRTWEKLLLAARAIVAIENPADVSVISSRNTGQRAVLKFAAATGATPIAGRFTPGTFTNQIQAAFREPRLLVVTDPRADHQPLTEASYVNLPTIALCNTDSPLRYVDIAIPCNNKGAHSVGLMWWMLAREVLRMRGTISREHPWEVMPDLYFYRDPEEIEKEEQAAAEKAVTKEEFQGEWTAPAPEFTATQPEVADWSEGVQVPSVPIQQFPTEDWSAQPATEDWSAAPTAQATEWVGATTDWS",
# 5HT1A_HUMAN
"MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTLIFCAVLGNACVVAAIALERSLQNVANYLIGSLAVTDLMVSVLVLPMAALYQVLNKWTLGQVTCDLFIALDVLCCTSSILHLCAIALDRYWAITDPIDYVNKRTPRRAAALISLTWLIGFLISIPPMLGWRTPEDRSDPDACTISKDHGYTIYSTFGAFYIPLLLMLVLYGRIFRAARFRIRKTVKKVEKTGADTRHGASPAPQPKKSVNGESGSRNWRLGVESKAGGALCANGAVRQGDDGAALEVIEVHRVGNSKEHLPLPSEAGPTPCAPASFERKNERNAEAKRKMALARERKTVKTLGIIMGTFILCWLPFFIVALVLPFCESSCHMPTLLGAIINWLGYSNSLLNPVIYAYFNKDFQNAFKKIIKCKFCRQ",
# ADA2A_HUMAN
"MFRQEQPLAEGSFAPMGSLQPDAGNASWNGTEAPGGGARATPYSLQVTLTLVCLAGLLMLLTVFGNVLVIIAVFTSRALKAPQNLFLVSLASADILVATLVIPFSLANEVMGYWYFGKAWCEIYLALDVLFCTSSIVHLCAISLDRYWSITQAIEYNLKRTPRRIKAIIITVWVISAVISFPPLISIEKKGGGGGPQPAEPRCEINDQKWYVISSCIGSFFAPCLIMILVYVRIYQIAKRRTRVPPSRRGPDAVAAPPGGTERRPNGLGPERSAGPGGAEAEPLPTQLNGAPGEPAPAGPRDTDALDLEESSSSDHAERPPGPRRPERGPRGKGKARASQVKPGDSLPRRGPGATGIGTPAAGPGEERVGAAKASRWRGRQNREKRFTFVLAVVIGVFVVCWFPFFFTYTLTAVGCSVPRTLFKFFFWFGYCNSSLNPVIYTIFNHDFRRAFKKILCRGDRKRIV",
# GNAO_HUMAN
"MGCTLSAEERAALERSKAIEKNLKEDGISAAKDVKLLLLGAGESGKSTIVKQMKIIHEDGFSGEDVKQYKPVVYSNTIQSLAAIVRAMDTLGIEYGDKERKADAKMVCDVVSRMEDTEPFSAELLSAMMRLWGDSGIQECFNRSREYQLNDSAKYYLDSLDRIGAADYQPTEQDILRTRVKTTGIVETHFTFKNLHFRLFDVGGQRSERKKWIHCFEDVTAIIFCVALSGYDQVLHEDETTNRMHESLMLFDSICNNKFFIDTSIILFLNKKDLFGEKIKKSPLTICFPEYTGPNTYEDAAAYIQAQFESKNRSPNKEIYCHMTCATDTNNIQVVFDAVTDIIIANNLRGCGLY",
# SPRC_HUMAN
"MRAWIFFLLCLAGRALAAPQQEALPDETEVVEETVAEVTEVSVGANPVQVEVGEFDDGAEETEEEVVAENPCQNHHCKHGKVCELDENNTPMCVCQDPTSCPAPIGEFEKVCSNDNKTFDSSCHFFATKCTLEGTKKGHKLHLDYIGPCKYIPPCLDSELTEFPLRMRDWLKNVLVTLYERDEDNNLLTEKQKLRVKKIHENEKRLEAGDHPVELLARDFEKNYNMYIFPVHWQFGQLDQHPIDGYLSHTELAPLRAPLIPMEHCTTRFFETCDLDNDKYIALDEWAGCFGIKQKDIDKDLVI",
# CLCA_HUMAN
"MAELDPFGAPAGAPGGPALGNGVAGAGEEDPAAAFLAQQESEIAGIENDEAFAILDGGAPGPQPHGEPPGGPDAVDGVMNGEYYQESNGPTDSYAAISQVDRLQSEPESIRKWREEQMERLEALDANSRKQEAEWKEKAIKELEEWYARQDEQLQKTKANNRVADEAFYKQPFADVIGYVTNINHPCYSLEQAAEEAFVNDIDESSPGTEWERVARLCDFNPKSSKQAKDVSRMRSVLISLKQAPLVH",
# CLCB_HUMAN
"MADDFGFFSSSESGAPEAAEEDPAAAFLAQQESEIAGIENDEGFGAPAGSHAAPAQPGPTSGAGSEDMGTTVNGDVFQEANGPADGYAAIAQADRLTQEPESIRKWREEQRKRLQELDAASKVTEQEWREKAKKDLEEWNQRQSEQVEKNKINNRIADKAFYQQPDADIIGYVASEEAFVKESKEETPGTEWEKVAQLCDFNPKSSKQCKDVSRLRSVLMSLKQTPLSR",
# ROA1_HUMAN
"MSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNARPHKVDGRVVEPKRAVSREDSQRPGAHLTVKKIFVGGIKEDTEEHHLRDYFEQYGKIEVIEIMTDRGSGKKRGFAFVTFDDHDSVDKIVIQKYHTVNGHNCEVRKALSKQEMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGGYGGGGPGYSGGSRGYGSGGQGYGNQGSGYGGSGSYDSYNNGGGGGFGGGSGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGPYGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF",
# ALDOC_HUMAN
"MPHSYPALSAEQKKELSDIALRIVAPGKGILAADESVGSMAKRLSQIGVENTEENRRLYRQVLFSADDRVKKCIGGVIFFHETLYQKDDNGVPFVRTIQDKGIVVGIKVDKGVVPLAGTDGETTTQGLDGLSERCAQYKKDGADFAKWRCVLKISERTPSALAILENANVLARYASICQQNGIVPIVEPEILPDGDHDLKRCQYVTEKVLAAVYKALSDHHVYLEGTLLKPNMVTPGHACPIKYTPEEIAMATVTALRRTVPPAVPGVTFLSGGQSEEEASFNLNAINRCPLPRPWALTFSYGRALQASALNAWRGQRDNAGAATEEFIKRAEVNGLAAQGKYEGSGEDGGAAAQSLYIANHAY",
# FBSP1_HUMAN
"MAAPAPGAGAASGGAGCSGGGAGAGAGSGSGAAGAGGRLPSRVLELVFSYLELSELRSCALVCKHWYRCLHGDENSEVWRSLCARSLAEEALRTDILCNLPSYKAKIRAFQHAFSTNDCSRNVYIKKNGFTLHRNPIAQSTDGARTKIGFSEGRHAWEVWWEGPLGTVAVIGIATKRAPMQCQGYVALLGSDDQSWGWNLVDNNLLHNGEVNGSFPQCNNAPKYQIGERIRVILDMEDKTLAFERGYEFLGVAFRGLPKVCLYPAVSAVYGNTEVTLVYLGKPLDG",
# ELFN1_HUMAN
"MAGRGWGALWVCVAAATLLHAGGLARADCWLIEGDKGFVWLAICSQNQPPYEAIPQQINSTIVDLRLNENRIRSVQYASLSRFGNLTYLNLTKNEIGYIEDGAFSGQFNLQVLQLGYNRLRNLTEGMLRGLGKLEYLYLQANLIEVVMASSFWECPNIVNIDLSMNRIQQLNSGTFAGLAKLSVCELYSNPFYCSCELLGFLRWLAAFTNATQTYDRMQCESPPVYSGYYLLGQGRRGHRSILSKLQSVCTEDSYAAEVVGPPRPASGRSQPGRSPPPPPPPEPSDMPCADDECFSGDGTTPLVALPTLATQAEARPLIKVKQLTQNSATITVQLPSPFHRMYTLEHFNNSKASTVSRLTKAQEEIRLTNLFTLTNYTYCVVSTSAGLRHNHTCLTICLPRLPSPPGPVPSPSTATHYIMTILGCLFGMVLVLGAVYYCLRRRRRQEEKHKKAASAAAAGSLKKTIIELKYGPELEAPGLAPLSQGPLLGPEAVTRIPYLPAAGEVEQYKLVESADTPKASKGSYMEVRTGDPPERRDCELGRPGPDSQSSVAEISTIAKEVDKVNQIINNCIDALKSESTSFQGVKSGPVSVAEPPLVLLSEPLAAKHGFLAPGYKDAFGHSLQRHHSVEAAGPPRASTSSSGSVRSPRAFRAEAVGVHKAAAAEAKYIEKGSPAADAILTVTPAAAVLRAEAEKGRQYGEHRHSYPGSHPAEPPAPPGPPPPPPHEGLGRKASILEPLTRPRPRDLAYSQLSPQYHSLSYSSSPEYTCRASQSIWERFRLSRRRHKEEEEFMAAGHALRKKVQFAKDEDLHDILDYWKGVSAQHKS",
# SRG2C_HUMAN
"MTSPAKFKKDKEIIAEYDTQVKEIRAQLTEQMKCLDQQCELRVQLLQDLQDFFRKKAEIEMDYSRNLEKLAEHFLAKTRSTKDQQFKKDQNVLSPVNCWNLLLNQVKWESRDHTTLSDIYLNNIIPRFVQVSEDSGRLFKKSKEVGQQLQDDLMKVLNELYSVMKTYHMYNADSISAQSKLKEAEKQEEKQIGKSVKQEDRQTPCSPDSTANVRIEEKHVRRSSVKKIEKMKEKHQAKYTENKLKAIKAQNEYLLALEATNASVFKYYIHDLSDLIDQCCDLGYHASLNRALRTFLSAELNLEQSKHEGLDAIENAVENLDATSDKQRLMEMYNNVFCPPMKFEFQPHMGDMASQLCAQQPVQSELVQRCQQLQSRLSTLKIENEEVKKTMEATLQTIQDIVTVEDFDVSDCFQYSNSMESVKSTVSETFMSKPSIAKRRANQQETEQFYFTVRECYGF",
# SRG2B_HUMAN
"MTSPAKFKKDKEIIAEYDTQVKEIRAQLTEQMKCLDQQCELRVQLLQDLQDFFRKKAEIEMDYSRNLEKLAERFLAKTCSTKDQQFKKDQNVLSPVNCWNLLLNQVKRESRDHTTLSDIYLNNIIPRFVQVSEDSGRLFKKSKEVGQQLQDDLMKVLNELYSVMKTYHMYNADSISAQSKLKEAEKQEEKQIGKSVKQEDRQTPRSPDSTANVRIEEKHVRRSSVKKIEKMKEKRQAKYTENKLKAIKARNEYLLALEATNASVFKYYIHDLSDLIDCCDLGYHASLNRALRTFLSAELNLEQSKHEGLDAIENAVENLDATSDKQRLMEMYNNVFCPPMKFEFQPHMGDMASQLCAQQPVQSELLQRCQQLQSRLSTLKIENEEVKKTMEATLQTIQDIVTVEDFDVSDCFQYSNSMESVKSTVSETFMSKPSIAKRRANQQETEQFYFTVRECYGF",
# AA3R_HUMAN
"MPNNSTALSLANVTYITMEIFIGLCAIVGNVLVICVVKLNPSLQTTTFYFIVSLALADIAVGVLVMPLAIVVSLGITIHFYSCLFMTCLLLIFTHASIMSLLAIAVDRYLRVKLTVRYKRVTTHRRIWLALGLCWLVSFLVGLTPMFGWNMKLTSEYHRNVTFLSCQFVSVMRMDYMVYFSFLTWIFIPLVVMCAIYLDIFYIIRNKLSLNLSNSKETGAFYGREFKTAKSLFLVLFLFALSWLPLSIINCIIYFNGEVPQLVLYMGILLSHANSMMNPIVYAYKIKKFKETYLLILKACVVCHPSDSLDTSIEKNSE",
# CALM1_HUMAN
"MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK",
# CALM2_HUMAN
"MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK",
# CALM3_HUMAN
"MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTEAELQDMINEVDADGNGTIDFPEFLTMMARKMKDTDSEEEIREAFRVFDKDGNGYISAAELRHVMTNLGEKLTDEEVDEMIREADIDGDGQVNYEEFVQMMTAK",
# RAP2A_HUMAN
"MREYKVVVLGSGGVGKSALTVQFVTGTFIEKYDPTIEDFYRKEIEVDSSPSVLEILDTAGTEQFASMRDLYIKNGQGFILVYSLVNQQSFQDIKPMRDQIIRVKRYEKVPVILVGNKVDLESEREVSSSEGRALAEEWGCPFMETSAKSKTMVDELFAEIVRQMNYAAQPDKDDPCCSACNIQ",
# PTPRF_HUMAN
"MAPEPAPGRTMVPLVPALVMLGLVAGAHGDSKPVFIKVPEDQTGLSGGVASFVCQATGEPKPRITWMKKGKKVSSQRFEVIEFDDGAGSVLRIQPLRVQRDEAIYECTATNSLGEINTSAKLSVLEEEQLPPGFPSIDMGPQLKVVEKARTATMLCAAGGNPDPEISWFKDFLPVDPATSNGRIKQLRSGALQIESSEESDQGKYECVATNSAGTRYSAPANLYVRVRRVAPRFSIPPSSQEVMPGGSVNLTCVAVGAPMPYVKWMMGAEELTKEDEMPVGRNVLELSNVVRSANYTCVAISSLGMIEATAQVTVKALPKPPIDLVVTETTATSVTLTWDSGNSEPVTYYGIQYRAAGTEGPFQEVDGVATTRYSIGGLSPFSEYAFRVLAVNSIGRGPPSEAVRARTGEQAPSSPPRRVQARMLSASTMLVQWEPPEEPNGLVRGYRVYYTPDSRRPPNAWHKHNTDAGLLTTVGSLLPGITYSLRVLAFTAVGDGPPSPTIQVKTQQGVPAQPADFQAEVESDTRIQLSWLLPPQERIIMYELVYWAAEDEDQQHKVTFDPTSSYTLEDLKPDTLYRFQLAARSDMGVGVFTPTIEARTAQSTPSAPPQKVMCVSMGSTTVRVSWVPPPADSRNGVITQYSVAYEAVDGEDRGRHVVDGISREHSSWDLVGLEKWTEYRVWVRAHTDVGPGPESSPVLVRTDEDVPSGPPRKVEVEPLNSTAVHVYWKLPVPSKQHGQIRGYQVTYVRLENGEPRGLPIIQDVMLAEAQWRPEESEDYETTISGLTPETTYSVTVAAYTTKGDGARSKPKIVTTTGAVPGRPTMMISTTAMNTALLQWHPPKELPGELLGYRLQYCRADEARPNTIDFGKDDQHFTVTGLHKGTTYIFRLAAKNRAGLGEEFEKEIRTPEDLPSGFPQNLHVTGLTTSTTELAWDPPVLAERNGRIISYTVVFRDINSQQELQNITTDTRFTLTGLKPDTTYDIKVRAWTSKGSGPLSPSIQSRTMPVEQVFAKNFRVAAAMKTSVLLSWEVPDSYKSAVPFKILYNGQSVEVDGHSMRKLIADLQPNTEYSFVLMNRGSSAGGLQHLVSIRTAPDLLPHKPLPASAYIEDGRFDLSMPHVQDPSLVRWFYIVVVPIDRVGGSMLTPRWSTPEELELDELLEAIEQGGEEQRRRRRQAERLKPYVAAQLDVLPETFTLGDKKNYRGFYNRPLSPDLSYQCFVLASLKEPMDQKRYASSPYSDEIVVQVTPAQQQEEPEMLWVTGPVLAVILIILIVIAILLFKRKRTHSPSSKDEQSIGLKDSLLAHSSDPVEMRRLNYQTPGMRDHPPIPITDLADNIERLKANDGLKFSQEYESIDPGQQFTWENSNLEVNKPKNRYANVIAYDHSRVILTSIDGVPGSDYINANYIDGYRKQNAYIATQGPLPETMGDFWRMVWEQRTATVVMMTRLEEKSRVKCDQYWPARGTETCGLIQVTLLDTVELATYTVRTFALHKSGSSEKRELRQFQFMAWPDHGVPEYPTPILAFLRRVKACNPLDAGPMVVHCSAGVGRTGCFIVIDAMLERMKHEKTVDIYGHVTCMRSQRNYMVQTEDQYVFIHEALLEAATCGHTEVPARNLYAHIQKLGQVPPGESVTAMELEFKLLASSKAHTSRFISANLPCNKFKNRLVNIMPYELTRVCLQPIRGVEGSDYINASFLDGYRQQKAYIATQGPLAESTEDFWRMLWEHNSTIIVMLTKLREMGREKCHQYWPAERSARYQYFVVDPMAEYNMPQYILREFKVTDARDGQSRTIRQFQFTDWPEQGVPKTGEGFIDFIGQVHKTKEQFGQDGPITVHCSAGVGRTGVFITLSIVLERMRYEGVVDMFQTVKTLRTQRPAMVQTEDQYQLCYRAALEYLGSFDHYAT",
# KAP0_HUMAN
"MESGSTAASEEARSLRECELYVQKHNIQALLKDSIVQLCTARPERPMAFLREYFERLEKEEAKQIQNLQKAGTRTDSREDEISPPPPNPVVKGRRRRGAISAEVYTEEDAASYVRKVIPKDYKTMAALAKAIEKNVLFSHLDDNERSDIFDAMFSVSFIAGETVIQQGDEGDNFYVIDQGETDVYVNNEWATSVGEGGSFGELALIYGTPRAATVKAKTNVKLWGIDRDSYRRILMGSTLRKRKMYEEFLSKVSILESLDKWERLTVADALEPVQFEDGQKIVVQGEPGDEFFIILEGSAAVLQRRSENEEFVEVGRLGPSDYFGEIALLMNRPRAATVVARGPLKCVKLDRPRFERVLGPCSDILKRNIQQYNSFVSLSV",
# CLUS_HUMAN
"MMKTLLLFVGLLLTWESGQVLGDQTVSDNELQEMSNQGSKYVNKEIQNAVNGVKQIKTLIEKTNEERKTLLSNLEEAKKKKEDALNETRESETKLKELPGVCNETMMALWEECKPCLKQTCMKFYARVCRSGSGLVGRQLEEFLNQSSPFYFWMNGDRIDSLLENDRQQTHMLDVMQDHFSRASSIIDELFQDRFFTREPQDTYHYLPFSLPHRRPHFFFPKSRIVRSLMPFSPYEPLNFHAMFQPFLEMIHEAQQAMDIHFHSPAFQHPPTEFIREGDDDRTVCREIRHNSTGCLRMKDQCDKCREILSVDCSTNNPSQAKLRRELDESLQVAERLTRKYNELLKSYQWKMLNTSSLLEQLNEQFNWVSRLANLTQGEDQYYLRVTTVASHTSDSDVPSGVTEVVVKLFDSDPITVTVPVEVSRKNPKFMETVAEKALQEYRKKHREE",
# HPLN1_HUMAN
"MKSLLLLVLISICWADHLSDNYTLDHDRAIHIQAENGPHLLVEAEQAKVFSHRGGNVTLPCKFYRDPTAFGSGIHKIRIKWTKLTSDYLKEVDVFVSMGYHKKTYGGYQGRVFLKGGSDSDASLVITDLTLEDYGRYKCEVIEGLEDDTVVVALDLQGVVFPYFPRLGRYNLNFHEAQQACLDQDAVIASFDQLYDAWRGGLDWCNAGWLSDGSVQYPITKPREPCGGQNTVPGVRNYGFWDKDKSRYDVFCFTSNFNGRFYYLIHPTKLTYDEAVQACLNDGAQIAKVGQIFAAWKILGYDRCDAGWLADGSVRYPISRPRRRCSPTEAAVRFVGFPDKKHKLYGVYCFRAYN",
# LAMC1_HUMAN
"MRGSHRAAPALRPRGRLWPVLAVLAAAAAAGCAQAAMDECTDEGGRPQRCMPEFVNAAFNVTVVATNTCGTPPEEYCVQTGVTGVTKSCHLCDAGQPHLQHGAAFLTDYNNQADTTWWQSQTMLAGVQYPSSINLTLHLGKAFDITYVRLKFHTSRPESFAIYKRTREDGPWIPYQYYSGSCENTYSKANRGFIRTGGDEQQALCTDEFSDISPLTGGNVAFSTLEGRPSAYNFDNSPVLQEWVTATDIRVTLNRLNTFGDEVFNDPKVLKSYYYAISDFAVGGRCKCNGHASECMKNEFDKLVCNCKHNTYGVDCEKCLPFFNDRPWRRATAESASECLPCDCNGRSQECYFDPELYRSTGHGGHCTNCQDNTDGAHCERCRENFFRLGNNEACSSCHCSPVGSLSTQCDSYGRCSCKPGVMGDKCDRCQPGFHSLTEAGCRPCSCDPSGSIDECNIETGRCVCKDNVEGFNCERCKPGFFNLESSNPRGCTPCFCFGHSSVCTNAVGYSVYSISSTFQIDEDGWRAEQRDGSEASLEWSSERQDIAVISDSYFPRYFIAPAKFLGKQVLSYGQNLSFSFRVDRRDTRLSAEDLVLEGAGLRVSVPLIAQGNSYPSETTVKYVFRLHEATDYPWRPALTPFEFQKLLNNLTSIKIRGTYSERSAGYLDDVTLASARPGPGVPATWVESCTCPVGYGGQFCEMCLSGYRRETPNLGPYSPCVLCACNGHSETCDPETGVCNCRDNTAGPHCEKCSDGYYGDSTAGTSSDCQPCPCPGGSSCAVVPKTKEVVCTNCPTGTTGKRCELCDDGYFGDPLGRNGPVRLCRLCQCSDNIDPNAVGNCNRLTGECLKCIYNTAGFYCDRCKDGFFGNPLAPNPADKCKACNCNLYGTMKQQSSCNPVTGQCECLPHVTGQDCGACDPGFYNLQSGQGCERCDCHALGSTNGQCDIRTGQCECQPGITGQHCERCEVNHFGFGPEGCKPCDCHPEGSLSLQCKDDGRCECREGFVGNRCDQCEENYFYNRSWPGCQECPACYRLVKDKVADHRVKLQELESLIANLGTGDEMVTDQAFEDRLKEAEREVMDLLREAQDVKDVDQNLMDRLQRVNNTLSSQISRLQNIRNTIEETGNLAEQARAHVENTERLIEIASRELEKAKVAAANVSVTQPESTGDPNNMTLLAEEARKLAERHKQEADDIVRVAKTANDTSTEAYNLLLRTLAGENQTAFEIEELNRKYEQAKNISQDLEKQAARVHEEAKRAGDKAVEIYASVAQLSPLDSETLENEANNIKMEAENLEQLIDQKLKDYEDLREDMRGKELEVKNLLEKGKTEQQTADQLLARADAAKALAEEAAKKGRDTLQEANDILNNLKDFDRRVNDNKTAAEEALRKIPAINQTITEANEKTREAQQALGSAAADATEAKNKAHEAERIASAVQKNATSTKAEAERTFAEVTDLDNEVNNMLKQLQEAEKELKRKQDDADQDMMMAGMASQAAQEAEINARKAKNSVTSLLSIINDLLEQLGQLDTVDLNKLNEIEGTLNKAKDEMKVSDLDRKVSDLENEAKKQEAAIMDYNRDIEEIMKDIRNLEDIRKTLPSGCFNTPSIEKP",
# MTAP2_HUMAN
"MADERKDEAKAPHWTSAPLTEASAHSHPPEIKDQGGAGEGLVRSANGFPYREDEEGAFGEHGSQGTYSNTKENGINGELTSADRETAEEVSARIVQVVTAEAVAVLKGEQEKEAQHKDQTAALPLAAEETANLPPSPPPSPASEQTVTVEEDLLTASKMEFHDQQELTPSTAEPSDQKEKESEKQSKPGEDLKHAALVSQPETTKTYPDKKDMQGTEEEKAPLALFGHTLVASLEDMKQKTEPSLVVPGIDLPKEPPTPKEQKDWFIEMPTEAKKDEWGLVAPISPGPLTPMREKDVFDDIPKWEGKQFDSPMPSPFQGGSFTLPLDVMKNEIVTETSPFAPAFLQPDDKKSLQQTSGPATAKDSFKIEEPHEAKPDKMAEAPPSEAMTLPKDAHIPVVEEHVMGKVLEEEKEAINQETVQQRDTFTPSGQEPILTEKETELKLEEKTTISDKEAVPKESKPPKPADEEIGIIQTSTEHTFSEQKDQEPTTDMLKQDSFPVSLEQAVTDSAMTSKTLEKAMTEPSALIEKSSIQELFEMRVDDKDKIEGVGAATSAELDMPFYEDKSGMSKYFETSALKEEATKSIEPGSDYYELSDTRESVHESIDTMSPMHKNGDKEFQTGKESQPSPPAQEAGYSTLAQSYPSDLPEEPSSPQERMFTIDPKVYGEKRDLHSKNKDDLTLSRSLGLGGRSAIEQRSMSINLPMSCLDSIALGFNFGRGHDLSPLASDILTNTSGSMDEGDDYLPATTPALEKAPCFPVESKEEEQIEKVKATGEESTQAEISCESPFLAKDFYKNGTVMAPDLPEMLDLAGTRSRLASVSADAEVARRKSVPSETVVEDSRTGLPPVTDENHVIVKTDSQLEDLGYCVFNKYTVPLPSPVQDSENLSGESGTFYEGTDDKVRRDLATDLSLIEVKLAAAGRVKDEFSVDKEASAHISGDKSGLSKEFDQEKKANDRLDTVLEKSEEHADSKEHAKKTEEAGDEIETFGLGVTYEQALAKDLSIPTDASSEKAEKGLSSVPEIAEVEPSKKVEQGLDFAVQGQLDVKISDFGQMASGLNIDDRRATELKLEATQDMTPSSKAPQEADAFMGVESGHMKEGTKVSETEVKEKVAKPDLVHQEAVDKEESYESSGEHESLTMESLKADEGKKETSPESSLIQDEIAVKLSVEIPCPPAVSEADLATDERADVQMEFIQGPKEESKETPDISITPSDVAEPLHETIVSEPAEIQSEEEEIEAQGEYDKLLFRSDTLQITDLGVSGAREEFVETCPSEHKGVIESVVTIEDDFITVVQTTTDEGESGSHSVRFAALEQPEVERRPSPHDEEEFEVEEAAEAQAEPKDGSPEAPASPEREEVALSEYKTETYDDYKDETTIDDSIMDADSLWVDTQDDDRSIMTEQLETIPKEEKAEKEARRSSLEKHRKEKPFKTGRGRISTPERKVAKKEPSTVSRDEVRRKKAVYKKAELAKKTEVQAHSPSRKFILKPAIKYTRPTHLSCVKRKTTAAGGESALAPSVFKQAKDKVSDGVTKSPEKRSSLPRPSSILPPRRGVSGDRDENSFSLNSSISSSARRTTRSEPIRRAGKSGTSTPTTPGSTAITPGTPPSYSSRTPGTPGTPSYPRTPHTPGTPKSAILVPSEKKVAIIRTPPKSPATPKQLRLINQPLPDLKNVKSKIGSTDNIKYQPKGGQVQIVTKKIDLSHVTSKCGSLKNIRHRPGGGRVKIESVKLDFKEKAQAKVGSLDNAHHVPGGGNVKIDSQKLNFREHAKARVDHGAEIITQSPGRSSVASPRRLSNVSSSGSINLLESPQLATLAEDVTAALAKQGL",
# HSP7C_HUMAN
"MSKGPAVGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVAMNPTNTVFDAKRLIGRRFDDAVVQSDMKHWPFMVVNDAGRPKVQVEYKGETKSFYPEEVSSMVLTKMKEIAEAYLGKTVTNAVVTVPAYFNDSQRQATKDAGTIAGLNVLRIINEPTAAAIAYGLDKKVGAERNVLIFDLGGGTFDVSILTIEDGIFEVKSTAGDTHLGGEDFDNRMVNHFIAEFKRKHKKDISENKRAVRRLRTACERAKRTLSSSTQASIEIDSLYEGIDFYTSITRARFEELNADLFRGTLDPVEKALRDAKLDKSQIHDIVLVGGSTRIPKIQKLLQDFFNGKELNKSINPDEAVAYGAAVQAAILSGDKSENVQDLLLLDVTPLSLGIETAGGVMTVLIKRNTTIPTKQTQTFTTYSDNQPGVLIQVYEGERAMTKDNNLLGKFELTGIPPAPRGVPQIEVTFDIDANGILNVSAVDKSTGKENKITITNDKGRLSKEDIERMVQEAEKYKAEDEKQRDKVSSKNSLESYAFNMKATVEDEKLQGKINDEDKQKILDKCNEIINWLDKNQTAEKEEFEHQQKELEKVCNPIITKLYQSAGGMPGGMPGGFPGGGAPPSGGASSGPTIEEVD",
# GTR3_HUMAN
"MGTQKVTPALIFAITVATIGSFQFGYNTGVINAPEKIIKEFINKTLTDKGNAPPSEVLLTSLWSLSVAIFSVGGMIGSFSVGLFVNRFGRRNSMLIVNLLAVTGGCFMGLCKVAKSVEMLILGRLVIGLFCGLCTGFVPMYIGEISPTALRGAFGTLNQLGIVVGILVAQIFGLEFILGSEELWPLLLGFTILPAILQSAALPFCPESPRFLLINRKEEENAKQILQRLWGTQDVSQDIQEMKDESARMSQEKQVTVLELFRVSSYRQPIIISIVLQLSQQLSGINAVFYYSTGIFKDAGVQEPIYATIGAGVVNTIFTVVSLFLVERAGRRTLHMIGLGGMAFCSTLMTVSLLLKDNYNGMSFVCIGAILVFVAFFEIGPGPIPWFIVAELFSQGPRPAAMAVAGCSNWTSNFLVGLLFPSAAHYLGAYVFIIFTGFLITFLAFTFFKVPETRGRTFEDITRAFEGQAHGADRSGKDGVMEMNSIEPAKETTTNV",
# ACM1_HUMAN
"MNTSAPPAVSPNITVLAPGKGPWQVAFIGITTGLLSLATVTGNLLVLISFKVNTELKTVNNYFLLSLACADLIIGTFSMNLYTTYLLMGHWALGTLACDLWLALDYVASNASVMNLLLISFDRYFSVTRPLSYRAKRTPRRAALMIGLAWLVSFVLWAPAILFWQYLVGERTVLAGQCYIQFLSQPIITFGTAMAAFYLPVTVMCTLYWRIYRETENRARELAALQGSETPGKGGGSSSSSERSQPGAEGSPETPPGRCCRCCRAPRLLQAYSWKEEEEEDEGSMESLTSSEGEEPGSEVVIKMPMVDPEAQAPTKQPPRSSPNTVKRPTKKGRDRAGKGQKPRGKEQLAKRKTFSLVKEKKAARTLSAILLAFILTWTPYNIMVLVSTFCKDCVPETLWELGYWLCYVNSTINPMCYALCNKAFRDTFRLLLLCRWDKRRWRKIPKRPGSVHRTPSRQC",
# ACHB_HUMAN
"MTPGALLMLLGALGAPLAPGVRGSEAEGRLREKLFSGYDSSVRPAREVGDRVRVSVGLILAQLISLNEKDEEMSTKVYLDLEWTDYRLSWDPAEHDGIDSLRITAESVWLPDVVLLNNNDGNFDVALDISVVVSSDGSVRWQPPGIYRSSCSIQVTYFPFDWQNCTMVFSSYSYDSSEVSLQTGLGPDGQGHQEIHIHEGTFIENGQWEIIHKPSRLIQPPGDPRGGREGQRQEVIFYLIIRRKPLFYLVNVIAPCILITLLAIFVFYLPPDAGEKMGLSIFALLTLTVFLLLLADKVPETSLSVPIIIKYLMFTMVLVTFSVILSVVVLNLHHRSPHTHQMPLWVRQIFIHKLPLYLRLKRPKPERDLMPEPPHCSSPGSGWGRGTDEYFIRKPPSDFLFPKPNRFQPELSAPDLRRFIDGPNRAVALLPELREVVSSISYIARQLQEQEDHDALKEDWQFVAMVVDRLFLWTFIIFTSVGTLVIFLDATYHLPPPDPFP",
# RALA_HUMAN
"MAANKPKGQNSLALHKVIMVGSGGVGKSALTLQFMYDEFVEDYEPTKADSYRKKVVLDGEEVQIDILDTAGQEDYAAIRDNYFRSGEGFLCVFSITEMESFAATADFREQILRVKEDENVPFLLVGNKSDLEDKRQVSVEEAKNRAEQWNVNYVETSAKTRANVDKVFFDLMREIRARKMEDSKEKNGKKKRKSLAKRIRERCCIL",
# BCR_HUMAN
"MVDPVGFAEAWKAQFPDSEPPRMELRSVGDIEQELERCKASIRRLEQEVNQERFRMIYLQTLLAKEKKSYDRQRWGFRRAAQAPDGASEPRASASRPQPAPADGADPPPAEEPEARPDGEGSPGKARPGTARRPGAAASGERDDRGPPASVAALRSNFERIRKGHGQPGADAEKPFYVNVEFHHERGLVKVNDKEVSDRISSLGSQAMQMERKKSQHGAGSSVGDASRPPYRGRSSESSCGVDGDYEDAELNPRFLKDNLIDANGGSRPPWPPLEYQPYQSIYVGGMMEGEGKGPLLRSQSTSEQEKRLTWPRRSYSPRSFEDCGGGYTPDCSSNENLTSSEEDFSSGQSSRVSPSPTTYRMFRDKSRSPSQNSQQSFDSSSPPTPQCHKRHRHCPVVVSEATIVGVRKTGQIWPNDGEGAFHGDADGSFGTPPGYGCAADRAEEQRRHQDGLPYIDDSPSSSPHLSSKGRGSRDALVSGALESTKASELDLEKGLEMRKWVLSGILASEETYLSHLEALLLPMKPLKAAATTSQPVLTSQQIETIFFKVPELYEIHKEFYDGLFPRVQQWSHQQRVGDLFQKLASQLGVYRAFVDNYGVAMEMAEKCCQANAQFAEISENLRARSNKDAKDPTTKNSLETLLYKPVDRVTRSTLVLHDLLKHTPASHPDHPLLQDALRISQNFLSSINEEITPRRQSMTVKKGEHRQLLKDSFMVELVEGARKLRHVFLFTDLLLCTKLKKQSGGKTQQYDCKWYIPLTDLSFQMVDELEAVPNIPLVPDEELDALKIKISQIKNDIQREKRANKGSKATERLKKKLSEQESLLLLMSPSMAFRVHSRNGKSYTFLISSDYERAEWRENIREQQKKCFRSFSLTSVELQMLTNSCVKLQTVHSIPLTINKEDDESPGLYGFLNVIVHSATGFKQSSNLYCTLEVDSFGYFVNKAKTRVYRDTAEPNWNEEFEIELEGSQTLRILCYEKCYNKTKIPKEDGESTDRLMGKGQVQLDPQALQDRDWQRTVIAMNGIEVKLSVKFNSREFSLKRMPSRKQTGVFGVKIAVVTKRERSKVPYIVRQCVEEIERRGMEEVGIYRVSGVATDIQALKAAFDVNNKDVSVMMSEMDVNAIAGTLKLYFRELPEPLFTDEFYPNFAEGIALSDPVAKESCMLNLLLSLPEANLLTFLFLLDHLKRVAEKEAVNKMSLHNLATVFGPTLLRPSEKESKLPANPSQPITMTDSWSLEVMSQVQVLLYFLQLEAIPAPDSKRQSILFSTEV",
# SPTB1_HUMAN
"MTSATEFENVGNQPPYSRINARWDAPDDELDNDNSSARLFERSRIKALADEREVVQKKTFTKWVNSHLARVSCRITDLYKDLRDGRMLIKLLEVLSGEMLPKPTKGKMRIHCLENVDKALQFLKEQRVHLENMGSHDIVDGNHRLVLGLIWTIILRFQIQDIVVQTQEGRETRSAKDALLLWCQMKTAGYPHVNVTNFTSSWKDGLAFNALIHKHRPDLIDFDKLKDSNARHNLEHAFNVAERQLGIIPLLDPEDVFTENPDEKSIITYVVAFYHYFSKMKVLAVEGKRVGKVIDHAIETEKMIEKYSGLASDLLTWIEQTITVLNSRKFANSLTGVQQQLQAFSTYRTVEKPPKFQEKGNLEVLLFTIQSRMRANNQKVYTPHDGKLVSDINRAWESLEEAEYRRELALRNELIRQEKLEQLARRFDRKAAMRETWLSENQRLVAQDNFGYDLAAVEAAKKKHEAIETDTAAYEERVRALEDLAQELEKENYHDQKRITARKDNILRLWSYLQELLQSRRQRLETTLALQKLFQDMLHSIDWMDEIKAHLLSAEFGKHLLEVEDLLQKHKLMEADIAIQGDKVKAITAATLKFTEGKGYQPCDPQVIQDRISHLEQCFEELSNMAAGRKAQLEQSKRLWKFFWEMDEAESWIKEKEQIYSSLDYGKDLTSVLILQRKHKAFEDELRGLDAHLEQIFQEAHGMVARKQFGHPQIEARIKEVSAQWDQLKDLAAFCKKNLQDAENFFQFQGDADDLKAWLQDAHRLLSGEDVGQDEGATRALGKKHKDFLEELEESRGVMEHLEQQAQGFPEEFRDSPDVTHRLQALRELYQQVVAQADLRQQRLQEALDLYTVFGETDACELWMGEKEKWLAEMEMPDTLEDLEVVQHRFDILDQEMKTLMTQIDGVNLAANSLVESGHPRSREVKQYQDHLNTRWQAFQTLVSERREAVDSALRVHNYCVDCEETSKWITDKTKVVESTKDLGRDLAGIIAIQRKLSGLERDVAAIQARVDALERESQQLMDSHPEQKEDIGQRQKHLEELWQGLQQSLQGQEDLLGEVSQLQAFLQDLDDFQAWLSITQKAVASEDMPESLPEAEQLLQQHAGIKDEIDGHQDSYQRVKESGEKVIQGQTDPEYLLLGQRLEGLDTGWNALGRMWESRSHTLAQCLGFQEFQKDAKQAEAILSNQEYTLAHLEPPDSLEAAEAGIRKFEDFLGSMENNRDKVLSPVDSGNKLVAEGNLYSDKIKEKVQLIEDRHRKNNEKAQEASVLLRDNLELQNFLQNCQELTLWINDKLLTSQDVSYDEARNLHNKWLKHQAFVAELASHEGWLENIDAEGKQLMDEKPQFTALVSQKLEALHRLWDELQATTKEKTQHLSAARSSDLRLQTHADLNKWISAMEDQLRSDDPGKDLTSVNRMLAKLKRVEDQVNVRKEELGELFAQVPSMGEEGGDADLSIEKRFLDLLEPLGRRKKQLESSRAKLQISRDLEDETLWVEERLPLAQSADYGTNLQTVQLFMKKNQTLQNEILGHTPRVEDVLQRGQQLVEAAEIDCQDLEERLGHLQSSWDRLREAAAGRLQRLRDANEAQQYYLDADEAEAWIGEQELYVISDEIPKDEEGAIVMLKRHLRQQRAVEDYGRNIKQLASRAQGLLSAGHPEGEQIIRLQGQVDKHYAGLKDVAEERKRKLENMYHLFQLKRETDDLEQWISEKELVASSPEMGQDFDHVTLLRDKFRDFARETGAIGQERVDNVNAFIERLIDAGHSEAATIAEWKDGLNEMWADLLELIDTRMQLLAASYDLHRYFYTGAEILGLIDEKHRELPEDVGLDASTAESFHRVHTAFERELHLLGVQVQQFQDVATRLQTAYAGEKAEAIQNKEQEVSAAWQALLDACAGRRTQLVDTADKFRFFSMARDLLSWMESIIRQIETQERPRDVSSVELLMKYHQGINAEIETRSKNFSACLELGESLLQRQHQASEEIREKLQQVMSRRKEMNEKWEARWERLRMLLEVCQFSRDASVAEAWLIAQEPYLASGDFGHTVDSVEKLIKRHEAFEKSTASWAERFAALEKPTTLELKERQIAERPAEETGPQEEEGETAGEAPVSHHAATERTSPVSLWSRLSSSWESLQPEPSHPY",
# LAMP1_HUMAN
"MAAPGSARRPLLLLLLLLLLGLMHCASAAMFMVKNGNGTACIMANFSAAFSVNYDTKSGPKNMTFDLPSDATVVLNRSSCGKENTSDPSLVIAFGRGHTLTLNFTRNATRYSVQLMSFVYNLSDTHLFPNASSKEIKTVESITDIRADIDKKYRCVSGTQVHMNNVTVTLHDATIQAYLSNSSFSRGETRCEQDRPSPTTAPPAPPSPSPSPVPKSPSVDKYNVSGTNGTCLLASMGLQLNLTYERKDNTTVTRLLNINPNKTSASGSCGAHLVTLELHSEGTTVLLFQFGMNASSSRFFLQGIQLNTILPDARDPAFKAANGSLRALQATVGNSYKCNAEEHVRVTKAFSVNIFKVWVQAFKVEGGQFGSVEECLLDENSMLIPIAVGGALAGLVLIVLIAYLVGRKRSHAGYQTI",
# FGFR1_HUMAN
"MWSWKCLLFWAVLVTATLCTARPSPTLPEQAQPWGAPVEVESFLVHPGDLLQLRCRLRDDVQSINWLRDGVQLAESNRTRITGEEVEVQDSVPADSGLYACVTSSPSGSDTTYFSVNVSDALPSSEDDDDDDDSSSEEKETDNTKPNRMPVAPYWTSPEKMEKKLHAVPAAKTVKFKCPSSGTPNPTLRWLKNGKEFKPDHRIGGYKVRYATWSIIMDSVVPSDKGNYTCIVENEYGSINHTYQLDVVERSPHRPILQAGLPANKTVALGSNVEFMCKVYSDPQPHIQWLKHIEVNGSKIGPDNLPYVQILKTAGVNTTDKEMEVLHLRNVSFEDAGEYTCLAGNSIGLSHHSAWLTVLEALEERPAVMTSPLYLEIIIYCTGAFLISCMVGSVIVYKMKSGTKKSDFHSQMAVHKLAKSIPLRRQVTVSADSSASMNSGVLLVRPSRLSSSGTPMLAGVSEYELPEDPRWELPRDRLVLGKPLGEGCFGQVVLAEAIGLDKDKPNRVTKVAVKMLKSDATEKDLSDLISEMEMMKMIGKHKNIINLLGACTQDGPLYVIVEYASKGNLREYLQARRPPGLEYCYNPSHNPEEQLSSKDLVSCAYQVARGMEYLASKKCIHRDLAARNVLVTEDNVMKIADFGLARDIHHIDYYKKTTNGRLPVKWMAPEALFDRIYTHQSDVWSFGVLLWEIFTLGGSPYPGVPVEELFKLLKEGHRMDKPSNCTNELYMMMRDCWHAVPSQRPTFKQLVEDLDRIVALTSNQEYLDLSMPLDQYSPSFPDTRSSTCSSGEDSVFSHEPLPEEPCLPRHPAQLANGGLKRR",
# DMD_HUMAN
"MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQDGRRLLDLLEGLTGQKLPKEKGSTRVHALNNVNKALRVLQNNNVDLVNIGSTDIVDGNHKLTLGLIWNIILHWQVKNVMKNIMAGLQQTNSEKILLSWVRQSTRNYPQVNVINFTTSWSDGLALNALIHSHRPDLFDWNSVVCQQSATQRLEHAFNIARYQLGIEKLLDPEDVDTTYPDKKSILMYITSLFQVLPQQVSIEAIQEVEMLPRPPKVTKEEHFQLHHQMHYSQQITVSLAQGYERTSSPKPRFKSYAYTQAAYVTTSDPTRSPFPSQHLEAPEDKSFGSSLMESEVNLDRYQTALEEVLSWLLSAEDTLQAQGEISNDVEVVKDQFHTHEGYMMDLTAHQGRVGNILQLGSKLIGTGKLSEDEETEVQEQMNLLNSRWECLRVASMEKQSNLHRVLMDLQNQKLKELNDWLTKTEERTRKMEEEPLGPDLEDLKRQVQQHKVLQEDLEQEQVRVNSLTHMVVVVDESSGDHATAALEEQLKVLGDRWANICRWTEDRWVLLQDILLKWQRLTEEQCLFSAWLSEKEDAVNKIHTTGFKDQNEMLSSLQKLAVLKADLEKKKQSMGKLYSLKQDLLSTLKNKSVTQKTEAWLDNFARCWDNLVQKLEKSTAQISQAVTTTQPSLTQTTVMETVTTVTTREQILVKHAQEELPPPPPQKKRQITVDSEIRKRLDVDITELHSWITRSEAVLQSPEFAIFRKEGNFSDLKEKVNAIEREKAEKFRKLQDASRSAQALVEQMVNEGVNADSIKQASEQLNSRWIEFCQLLSERLNWLEYQNNIIAFYNQLQQLEQMTTTAENWLKIQPTTPSEPTAIKSQLKICKDEVNRLSDLQPQIERLKIQSIALKEKGQGPMFLDADFVAFTNHFKQVFSDVQAREKELQTIFDTLPPMRYQETMSAIRTWVQQSETKLSIPQLSVTDYEIMEQRLGELQALQSSLQEQQSGLYYLSTTVKEMSKKAPSEISRKYQSEFEEIEGRWKKLSSQLVEHCQKLEEQMNKLRKIQNHIQTLKKWMAEVDVFLKEEWPALGDSEILKKQLKQCRLLVSDIQTIQPSLNSVNEGGQKIKNEAEPEFASRLETELKELNTQWDHMCQQVYARKEALKGGLEKTVSLQKDLSEMHEWMTQAEEEYLERDFEYKTPDELQKAVEEMKRAKEEAQQKEAKVKLLTESVNSVIAQAPPVAQEALKKELETLTTNYQWLCTRLNGKCKTLEEVWACWHELLSYLEKANKWLNEVEFKLKTTENIPGGAEEISEVLDSLENLMRHSEDNPNQIRILAQTLTDGGVMDELINEELETFNSRWRELHEEAVRRQKLLEQSIQSAQETEKSLHLIQESLTFIDKQLAAYIADKVDAAQMPQEAQKIQSDLTSHEISLEEMKKHNQGKEAAQRVLSQIDVAQKKLQDVSMKFRLFQKPANFEQRLQESKMILDEVKMHLPALETKSVEQEVVQSQLNHCVNLYKSLSEVKSEVEMVIKTGRQIVQKKQTENPKELDERVTALKLHYNELGAKVTERKQQLEKCLKLSRKMRKEMNVLTEWLAATDMELTKRSAVEGMPSNLDSEVAWGKATQKEIEKQKVHLKSITEVGEALKTVLGKKETLVEDKLSLLNSNWIAVTSRAEEWLNLLLEYQKHMETFDQNVDHITKWIIQADTLLDESEKKKPQQKEDVLKRLKAELNDIRPKVDSTRDQAANLMANRGDHCRKLVEPQISELNHRFAAISHRIKTGKASIPLKELEQFNSDIQKLLEPLEAEIQQGVNLKEEDFNKDMNEDNEGTVKELLQRGDNLQQRITDERKREEIKIKQQLLQTKHNALKDLRSQRRKKALEISHQWYQYKRQADDLLKCLDDIEKKLASLPEPRDERKIKEIDRELQKKKEELNAVRRQAEGLSEDGAAMAVEPTQIQLSKRWREIESKFAQFRRLNFAQIHTVREETMMVMTEDMPLEISYVPSTYLTEITHVSQALLEVEQLLNAPDLCAKDFEDLFKQEESLKNIKDSLQQSSGRIDIIHSKKTAALQSATPVERVKLQEALSQLDFQWEKVNKMYKDRQGRFDRSVEKWRRFHYDIKIFNQWLTEAEQFLRKTQIPENWEHAKYKWYLKELQDGIGQRQTVVRTLNATGEEIIQQSSKTDASILQEKLGSLNLRWQEVCKQLSDRKKRLEEQKNILSEFQRDLNEFVLWLEEADNIASIPLEPGKEQQLKEKLEQVKLLVEELPLRQGILKQLNETGGPVLVSAPISPEEQDKLENKLKQTNLQWIKVSRALPEKQGEIEAQIKDLGQLEKKLEDLEEQLNHLLLWLSPIRNQLEIYNQPNQEGPFDVKETEIAVQAKQPDVEEILSKGQHLYKEKPATQPVKRKLEDLSSEWKAVNRLLQELRAKQPDLAPGLTTIGASPTQTVTLVTQPVVTKETAISKLEMPSSLMLEVPALADFNRAWTELTDWLSLLDQVIKSQRVMVGDLEDINEMIIKQKATMQDLEQRRPQLEELITAAQNLKNKTSNQEARTIITDRIERIQNQWDEVQEHLQNRRQQLNEMLKDSTQWLEAKEEAEQVLGQARAKLESWKEGPYTVDAIQKKITETKQLAKDLRQWQTNVDVANDLALKLLRDYSADDTRKVHMITENINASWRSIHKRVSEREAALEETHRLLQQFPLDLEKFLAWLTEAETTANVLQDATRKERLLEDSKGVKELMKQWQDLQGEIEAHTDVYHNLDENSQKILRSLEGSDDAVLLQRRLDNMNFKWSELRKKSLNIRSHLEASSDQWKRLHLSLQELLVWLQLKDDELSRQAPIGGDFPAVQKQNDVHRAFKRELKTKEPVIMSTLETVRIFLTEQPLEGLEKLYQEPRELPPEERAQNVTRLLRKQAEEVNTEWEKLNLHSADWQRKIDETLERLRELQEATDELDLKLRQAEVIKGSWQPVGDLLIDSLQDHLEKVKALRGEIAPLKENVSHVNDLARQLTTLGIQLSPYNLSTLEDLNTRWKLLQVAVEDRVRQLHEAHRDFGPASQHFLSTSVQGPWERAISPNKVPYYINHETQTTCWDHPKMTELYQSLADLNNVRFSAYRTAMKLRRLQKALCLDLLSLSAACDALDQHNLKQNDQPMDILQIINCLTTIYDRLEQEHNNLVNVPLCVDMCLNWLLNVYDTGRTGRIRVLSFKTGIISLCKAHLEDKYRYLFKQVASSTGFCDQRRLGLLLHDSIQIPRQLGEVASFGGSNIEPSVRSCFQFANNKPEIEAALFLDWMRLEPQSMVWLPVLHRVAAAETAKHQAKCNICKECPIIGFRYRSLKHFNYDICQSCFFSGRVAKGHKMHYPMVEYCTPTTSGEDVRDFAKVLKNKFRTKRYFAKHPRMGYLPVQTVLEGDNMETPVTLINFWPVDSAPASSPQLSHDDTHSRIEHYASRLAEMENSNGSYLNDSISPNESIDDEHLLIQHYCQSLNQDSPLSQPRSPAQILISLESEERGELERILADLEEENRNLQAEYDRLKQQHEHKGLSPLPSPPEMMPTSPQSPRDAELIAEAKLLRQHKGRLEARMQILEDHNKQLESQLHRLRQLLEQPQAEAKVNGTTVSSPSTSLQRSDSSQPMLLRVVGSQTSDSMGEEDLLSPPQDTSTGLEEVMEQLNNSFPSSRGRNTPGKPMREDTM",
# PABP1_HUMAN
"MNPSAPSYPMASLYVGDLHPDVTEAMLYEKFSPAGPILSIRVCRDMITRRSLGYAYVNFQQPADAERALDTMNFDVIKGKPVRIMWSQRDPSLRKSGVGNIFIKNLDKSIDNKALYDTFSAFGNILSCKVVCDENGSKGYGFVHFETQEAAERAIEKMNGMLLNDRKVFVGRFKSRKEREAELGARAKEFTNVYIKNFGEDMDDERLKDLFGKFGPALSVKVMTDESGKSKGFGFVSFERHEDAQKAVDEMNGKELNGKQIYVGRAQKKVERQTELKRKFEQMKQDRITRYQGVNLYVKNLDDGIDDERLRKEFSPFGTITSAKVMMEGGRSKGFGFVCFSSPEEATKAVTEMNGRIVATKPLYVALAQRKEERQAHLTNQYMQRMASVRAVPNPVINPYQPAPPSGYFMAAIPQTQNRAAYYPPSQIAQLRPSPRWTAQGARPHPFQNMPGAIRPAAPRPPFSTMRPASSQVPRVMSTQRVANTSTQTMGPRPAAAAAAATPAVRTVPQYKYAAGVRNPQQHLNAQPQVTMQQPAVHVQGQEPLTASMLASAPPQEQKQMLGERLFPLIQAMHPTLAGKITGMLLEIDNSELLHMLESPESLRSKVDEAVAVLQAHQAKEAAQKAVNSATGVPTV",
# NFH_HUMAN
"MMSFGGADALLGAPFAPLHGGGSLHYALARKGGAGGTRSAAGSSSGFHSWTRTSVSSVSASPSRFRGAGAASSTDSLDTLSNGPEGCMVAVATSRSEKEQLQALNDRFAGYIDKVRQLEAHNRSLEGEAAALRQQQAGRSAMGELYEREVREMRGAVLRLGAARGQLRLEQEHLLEDIAHVRQRLDDEARQREEAEAAARALARFAQEAEAARVDLQKKAQALQEECGYLRRHHQEEVGELLGQIQGSGAAQAQMQAETRDALKCDVTSALREIRAQLEGHAVQSTLQSEEWFRVRLDRLSEAAKVNTDAMRSAQEEITEYRRQLQARTTELEALKSTKDSLERQRSELEDRHQADIASYQEAIQQLDAELRNTKWEMAAQLREYQDLLNVKMALDIEIAAYRKLLEGEECRIGFGPIPFSLPEGLPKIPSVSTHIKVKSEEKIKVVEKSEKETVIVEEQTEETQVTEEVTEEEEKEAKEEEGKEEEGGEEEEAEGGEEETKSPPAEEAASPEKEAKSPVKEEAKSPAEAKSPEKEEAKSPAEVKSPEKAKSPAKEEAKSPPEAKSPEKEEAKSPAEVKSPEKAKSPAKEEAKSPAEAKSPEKAKSPVKEEAKSPAEAKSPVKEEAKSPAEVKSPEKAKSPTKEEAKSPEKAKSPEKEEAKSPEKAKSPVKAEAKSPEKAKSPVKAEAKSPEKAKSPVKEEAKSPEKAKSPVKEEAKSPEKAKSPVKEEAKTPEKAKSPVKEEAKSPEKAKSPEKAKTLDVKSPEAKTPAKEEARSPADKFPEKAKSPVKEEVKSPEKAKSPLKEDAKAPEKEIPKKEEVKSPVKEEEKPQEVKVKEPPKKAEEEKAPATPKTEEKKDSKKEEAPKKEAPKPKVEEKKEPAVEKPKESKVEAKKEEAEDKKKVPTPEKEAPAKVEVKEDAKPKEKTEVAKKEPDDAKAKEPSKPAEKKEAAPEKKDTKEEKAKKPEEKPKTEAKAKEDDKTLSKEPSKPKAEKAEKSSSTDQKDSKPPEKATEDKAAKGK",
# ACTN1_HUMAN
"MDHYDSQQTNDYMQPEEDWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIENIEEDFRDGLKLMLLLEVISGERLAKPERGKMRVHKISNVNKALDFIASKGVKLVSIGAEEIVDGNVKMTLGMIWTIILRFAIQDISVEETSAKEGLLLWCQRKTAPYKNVNIQNFHISWKDGLGFCALIHRHRPELIDYGKLRKDDPLTNLNTAFDVAEKYLDIPKMLDAEDIVGTARPDEKAIMTYVSSFYHAFSGAQKAETAANRICKVLAVNQENEQLMEDYEKLASDLLEWIRRTIPWLENRVPENTMHAMQQKLEDFRDYRRLHKPPKVQEKCQLEINFNTLQTKLRLSNRPAFMPSEGRMVSDINNAWGCLEQVEKGYEEWLLNEIRRLERLDHLAEKFRQKASIHEAWTDGKEAMLRQKDYETATLSEIKALLKKHEAFESDLAAHQDRVEQIAAIAQELNELDYYDSPSVNARCQKICDQWDNLGALTQKRREALERTEKLLETIDQLYLEYAKRAAPFNNWMEGAMEDLQDTFIVHTIEEIQGLTTAHEQFKATLPDADKERLAILGIHNEVSKIVQTYHVNMAGTNPYTTITPQEINGKWDHVRQLVPRRDQALTEEHARQQHNERLRKQFGAQANVIGPWIQTKMEEIGRISIEMHGTLEDQLSHLRQYEKSIVNYKPKIDQLEGDHQLIQEALIFDNKHTNYTMEHIRVGWEQLLTTIARTINEVENQILTRDAKGISQEQMNEFRASFNHFDRDHSGTLGPEEFKACLISLGYDIGNDPQGEAEFARIMSIVDPNRLGVVTFQAFIDFMSRETADTDTADQVMASFKILAGDKNYITMDELRRELPPDQAEYCIARMAPYTGPDSVPGALDYMSFSTALYGESDL",
# CADH1_HUMAN
"MGPWSRSLSALLLLLQVSSWLCQEPEPCHPGFDAESYTFTVPRRHLERGRVLGRVNFEDCTGRQRTAYFSLDTRFKVGTDGVITVKRPLRFHNPQIHFLVYAWDSTYRKFSTKVTLNTVGHHHRPPPHQASVSGIQAELLTFPNSSPGLRRQKRDWVIPPISCPENEKGPFPKNLVQIKSNKDKEGKVFYSITGQGADTPPVGVFIIERETGWLKVTEPLDRERIATYTLFSHAVSSNGNAVEDPMEILITVTDQNDNKPEFTQEVFKGSVMEGALPGTSVMEVTATDADDDVNTYNAAIAYTILSQDPELPDKNMFTINRNTGVISVVTTGLDRESFPTYTLVVQAADLQGEGLSTTATAVITVTDTNDNPPIFNPTTYKGQVPENEANVVITTLKVTDADAPNTPAWEAVYTILNDDGGQFVVTTNPVNNDGILKTAKGLDFEAKQQYILHVAVTNVVPFEVSLTTSTATVTVDVLDVNEAPIFVPPEKRVEVSEDFGVGQEITSYTAQEPDTFMEQKITYRIWRDTANWLEINPDTGAISTRAELDREDFEHVKNSTYTALIIATDNGSPVATGTGTLLLILSDVNDNAPIPEPRTIFFCERNPKPQVINIIDADLPPNTSPFTAELTHGASANWTIQYNDPTQESIILKPKMALEVGDYKINLKLMDNQNKDQVTTLEVSVCDCEGAAGVCRKAQPVEAGLQIPAILGILGGILALLILILLLLLFLRRRAVVKEPLLPPEDDTRDNVYYYDEEGGGEEDQDFDLSQLHRGLDARPEVTRNDVAPTLMSVPRYLPRPANPDEIGNFIDENLKAADTDPTAPPYDSLLVFDYEGSGSEAASLSSLNSSESDKDQDYDYLNEWGNRFKKLADMYGGGEDD",
# SRC_HUMAN
"MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL",
# SCG2_HUMAN
"MAEAKTHWLGAALSLIPLIFLISGAEAASFQRNQLLQKEPDLRLENVQKFPSPEMIRALEYIENLRQQAHKEESSPDYNPYQGVSVPLQQKENGDESHLPERDSLSEEDWMRIILEALRQAENEPQSAPKENKPYALNSEKNFPMDMSDDYETQQWPERKLKHMQFPPMYEENSRDNPFKRTNEIVEEQYTPQSLATLESVFQELGKLTGPNNQKRERMDEEQKLYTDDEDDIYKANNIAYEDVVGGEDWNPVEEKIESQTQEEVRDSKENIEKNEQINDEMKRSGQLGIQEEDLRKESKDQLSDDVSKVIAYLKRLVNAAGSGRLQNGQNGERATRLFEKPLDSQSIYQLIEISRNLQIPPEDLIEMLKTGEKPNGSVEPERELDLPVDLDDISEADLDHPDLFQNRMLSKSGYPKTPGRAGTEALPDGLSVEDILNLLGMESAANQKTSYFPNPYNQEKVLPRLPYGAGRSRSNQLPKAAWIPHVENRQMAYENLNDKDQELGEYLARMLVKYPEIINSNQVKRVPGQGSSEDDLQEEEQIEQAIKEHLNQGSSQETDKLAPVSKRFPVGPPKNDDTPNRQYWDEDLLMKVLEYLNQEKAEKGREHIAKRAMENM",
# NCAM1_HUMAN
"MLQTKDLIWTLFFLGTAVSLQVDIVPSQGEISVGESKFFLCQVAGDAKDKDISWFSPNGEKLTPNQQRISVVWNDDSSSTLTIYNANIDDAGIYKCVVTGEDGSESEATVNVKIFQKLMFKNAPTPQEFREGEDAVIVCDVVSSLPPTIIWKHKGRDVILKKDVRFIVLSNNYLQIRGIKKTDEGTYRCEGRILARGEINFKDIQVIVNVPPTIQARQNIVNATANLGQSVTLVCDAEGFPEPTMSWTKDGEQIEQEEDDEKYIFSDDSSQLTIKKVDKNDEAEYICIAENKAGEQDATIHLKVFAKPKITYVENQTAMELEEQVTLTCEASGDPIPSITWRTSTRNISSEEKASWTRPEKQETLDGHMVVRSHARVSSLTLKSIQYTDAGEYICTASNTIGQDSQSMYLEVQYAPKLQGPVAVYTWEGNQVNITCEVFAYPSATISWFRDGQLLPSSNYSNIKIYNTPSASYLEVTPDSENDFGNYNCTAVNRIGQESLEFILVQADTPSSPSIDQVEPYSSTAQVQFDEPEATGGVPILKYKAEWRAVGEEVWHSKWYDAKEASMEGIVTIVGLKPETTYAVRLAALNGKGLGEISAASEFKTQPVQGEPSAPKLEGQMGEDGNSIKVNLIKQDDGGSPIRHYLVRYRALSSEWKPEIRLPSGSDHVMLKSLDWNAEYEVYVVAENQQGKSKAAHFVFRTSAQPTAIPANGSPTSGLSTGAIVGILIVIFVLLLVVVDITCYFLNKCGLFMCIAVNLCGKAGPGAKGKDMEEGKAAFSKDESKEPIVEVRTEEERTPNHDGGKHTEPNETTPLTEPEKGPVEAKPECQETETKPAPAEVKTVPNDATQTKENESKA",
# CSPG2_HUMAN
"MFINIKSILWMCSTLIVTHALHKVKVGKSPPVRGSLSGKVSLPCHFSTMPTLPPSYNTSEFLRIKWSKIEVDKNGKDLKETTVLVAQNGNIKIGQDYKGRVSVPTHPEAVGDASLTVVKLLASDAGLYRCDVMYGIEDTQDTVSLTVDGVVFHYRAATSRYTLNFEAAQKACLDVGAVIATPEQLFAAYEDGFEQCDAGWLADQTVRYPIRAPRVGCYGDKMGKAGVRTYGFRSPQETYDVYCYVDHLDGDVFHLTVPSKFTFEEAAKECENQDARLATVGELQAAWRNGFDQCDYGWLSDASVRHPVTVARAQCGGGLLGVRTLYRFENQTGFPPPDSRFDAYCFKPKEATTIDLSILAETASPSLSKEPQMVSDRTTPIIPLVDELPVIPTEFPPVGNIVSFEQKATVQPQAITDSLATKLPTPTGSTKKPWDMDDYSPSASGPLGKLDISEIKEEVLQSTTGVSHYATDSWDGVVEDKQTQESVTQIEQIEVGPLVTSMEILKHIPSKEFPVTETPLVTARMILESKTEKKMVSTVSELVTTGHYGFTLGEEDDEDRTLTVGSDESTLIFDQIPEVITVSKTSEDTIHTHLEDLESVSASTTVSPLIMPDNNGSSMDDWEERQTSGRITEEFLGKYLSTTPFPSQHRTEIELFPYSGDKILVEGISTVIYPSLQTEMTHRRERTETLIPEMRTDTYTDEIQEEITKSPFMGKTEEEVFSGMKLSTSLSEPIHVTESSVEMTKSFDFPTLITKLSAEPTEVRDMEEDFTATPGTTKYDENITTVLLAHGTLSVEAATVSKWSWDEDNTTSKPLESTEPSASSKLPPALLTTVGMNGKDKDIPSFTEDGADEFTLIPDSTQKQLEEVTDEDIAAHGKFTIRFQPTTSTGIAEKSTLRDSTTEEKVPPITSTEGQVYATMEGSALGEVEDVDLSKPVSTVPQFAHTSEVEGLAFVSYSSTQEPTTYVDSSHTIPLSVIPKTDWGVLVPSVPSEDEVLGEPSQDILVIDQTRLEATISPETMRTTKITEGTTQEEFPWKEQTAEKPVPALSSTAWTPKEAVTPLDEQEGDGSAYTVSEDELLTGSERVPVLETTPVGKIDHSVSYPPGAVTEHKVKTDEVVTLTPRIGPKVSLSPGPEQKYETEGSSTTGFTSSLSPFSTHITQLMEETTTEKTSLEDIDLGSGLFEKPKATELIEFSTIKVTVPSDITTAFSSVDRLHTTSAFKPSSAITKKPPLIDREPGEETTSDMVIIGESTSHVPPTTLEDIVAKETETDIDREYFTTSSPPATQPTRPPTVEDKEAFGPQALSTPQPPASTKFHPDINVYIIEVRENKTGRMSDLSVIGHPIDSESKEDEPCSEETDPVHDLMAEILPEFPDIIEIDLYHSEENEEEEEECANATDVTTTPSVQYINGKHLVTTVPKDPEAAEARRGQFESVAPSQNFSDSSESDTHPFVIAKTELSTAVQPNESTETTESLEVTWKPETYPETSEHFSGGEPDVFPTVPFHEEFESGTAKKGAESVTERDTEVGHQAHEHTEPVSLFPEESSGEIAIDQESQKIAFARATEVTFGEEVEKSTSVTYTPTIVPSSASAYVSEEEAVTLIGNPWPDDLLSTKESWVEATPRQVVELSGSSSIPITEGSGEAEEDEDTMFTMVTDLSQRNTTDTLITLDTSRIITESFFEVPATTIYPVSEQPSAKVVPTKFVSETDTSEWISSTTVEEKKRKEEEGTTGTASTFEVYSSTQRSDQLILPFELESPNVATSSDSGTRKSFMSLTTPTQSEREMTDSTPVFTETNTLENLGAQTTEHSSIHQPGVQEGLTTLPRSPASVFMEQGSGEAAADPETTTVSSFSLNVEYAIQAEKEVAGTLSPHVETTFSTEPTGLVLSTVMDRVVAENITQTSREIVISERLGEPNYGAEIRGFSTGFPLEEDFSGDFREYSTVSHPIAKEETVMMEGSGDAAFRDTQTSPSTVPTSVHISHISDSEGPSSTMVSTSAFPWEEFTSSAEGSGEQLVTVSSSVVPVLPSAVQKFSGTASSIIDEGLGEVGTVNEIDRRSTILPTAEVEGTKAPVEKEEVKVSGTVSTNFPQTIEPAKLWSRQEVNPVRQEIESETTSEEQIQEEKSFESPQNSPATEQTIFDSQTFTETELKTTDYSVLTTKKTYSDDKEMKEEDTSLVNMSTPDPDANGLESYTTLPEATEKSHFFLATALVTESIPAEHVVTDSPIKKEESTKHFPKGMRPTIQESDTELLFSGLGSGEEVLPTLPTESVNFTEVEQINNTLYPHTSQVESTSSDKIEDFNRMENVAKEVGPLVSQTDIFEGSGSVTSTTLIEILSDTGAEGPTVAPLPFSTDIGHPQNQTVRWAEEIQTSRPQTITEQDSNKNSSTAEINETTTSSTDFLARAYGFEMAKEFVTSAPKPSDLYYEPSGEGSGEVDIVDSFHTSATTQATRQESSTTFVSDGSLEKHPEVPSAKAVTADGFPTVSVMLPLHSEQNKSSPDPTSTLSNTVSYERSTDGSFQDRFREFEDSTLKPNRKKPTENIIIDLDKEDKDLILTITESTILEILPELTSDKNTIIDIDHTKPVYEDILGMQTDIDTEVPSEPHDSNDESNDDSTQVQEIYEAAVNLSLTEETFEGSADVLASYTQATHDESMTYEDRSQLDHMGFHFTTGIPAPSTETELDVLLPTATSLPIPRKSATVIPEIEGIKAEAKALDDMFESSTLSDGQAIADQSEIIPTLGQFERTQEEYEDKKHAGPSFQPEFSSGAEEALVDHTPYLSIATTHLMDQSVTEVPDVMEGSNPPYYTDTTLAVSTFAKLSSQTPSSPLTIYSGSEASGHTEIPQPSALPGIDVGSSVMSPQDSFKEIHVNIEATFKPSSEEYLHITEPPSLSPDTKLEPSEDDGKPELLEEMEASPTELIAVEGTEILQDFQNKTDGQVSGEAIKMFPTIKTPEAGTVITTADEIELEGATQWPHSTSASATYGVEAGVVPWLSPQTSERPTLSSSPEINPETQAALIRGQDSTIAASEQQVAARILDSNDQATVNPVEFNTEVATPPFSLLETSNETDFLIGINEESVEGTAIYLPGPDRCKMNPCLNGGTCYPTETSYVCTCVPGYSGDQCELDFDECHSNPCRNGATCVDGFNTFRCLCLPSYVGALCEQDTETCDYGWHKFQGQCYKYFAHRRTWDAAERECRLQGAHLTSILSHEEQMFVNRVGHDYQWIGLNDKMFEHDFRWTDGSTLQYENWRPNQPDSFFSAGEDCVVIIWHENGQWNDVPCNYHLTYTCKKGTVACGQPPVVENAKTFGKMKPRYEINSLIRYHCKDGFIQRHLPTIRCLGNGRWAIPKITCMNPSAYQRTYSMKYFKNSSSAKDNSINTSKHDHRWSRRWQESRR",
# AT1A3_HUMAN
"MGDKKDDKDSPKKNKGKERRDLDDLKKEVAMTEHKMSVEEVCRKYNTDCVQGLTHSKAQEILARDGPNALTPPPTTPEWVKFCRQLFGGFSILLWIGAILCFLAYGIQAGTEDDPSGDNLYLGIVLAAVVIITGCFSYYQEAKSSKIMESFKNMVPQQALVIREGEKMQVNAEEVVVGDLVEIKGGDRVPADLRIISAHGCKVDNSSLTGESEPQTRSPDCTHDNPLETRNITFFSTNCVEGTARGVVVATGDRTVMGRIATLASGLEVGKTPIAIEIEHFIQLITGVAVFLGVSFFILSLILGYTWLEAVIFLIGIIVANVPEGLLATVTVCLTLTAKRMARKNCLVKNLEAVETLGSTSTICSDKTGTLTQNRMTVAHMWFDNQIHEADTTEDQSGTSFDKSSHTWVALSHIAGLCNRAVFKGGQDNIPVLKRDVAGDASESALLKCIELSSGSVKLMRERNKKVAEIPFNSTNKYQLSIHETEDPNDNRYLLVMKGAPERILDRCSTILLQGKEQPLDEEMKEAFQNAYLELGGLGERVLGFCHYYLPEEQFPKGFAFDCDDVNFTTDNLCFVGLMSMIDPPRAAVPDAVGKCRSAGIKVIMVTGDHPITAKAIAKGVGIISEGNETVEDIAARLNIPVSQVNPRDAKACVIHGTDLKDFTSEQIDEILQNHTEIVFARTSPQQKLIIVEGCQRQGAIVAVTGDGVNDSPALKKADIGVAMGIAGSDVSKQAADMILLDDNFASIVTGVEEGRLIFDNLKKSIAYTLTSNIPEITPFLLFIMANIPLPLGTITILCIDLGTDMVPAISLAYEAAESDIMKRQPRNPRTDKLVNERLISMAYGQIGMIQALGGFFSYFVILAENGFLPGNLVGIRLNWDDRTVNDLEDSYGQQWTYEQRKVVEFTCHTAFFVSIVVVQWADLIICKTRRNSVFQQGMKNKILIFGLFEETALAAFLSYCPGMDVALRMYPLKPSWWFCAFPYSFLIFVYDEIRKLILRRNPGGWVEKETYY",
# EF2_HUMAN
"MVNFTVDQIRAIMDKKANIRNMSVIAHVDHGKSTLTDSLVCKAGIIASARAGETRFTDTRKDEQERCITIKSTAISLFYELSENDLNFIKQSKDGAGFLINLIDSPGHVDFSSEVTAALRVTDGALVVVDCVSGVCVQTETVLRQAIAERIKPVLMMNKMDRALLELQLEPEELYQTFQRIVENVNVIISTYGEGESGPMGNIMIDPVLGTVGFGSGLHGWAFTLKQFAEMYVAKFAAKGEGQLGPAERAKKVEDMMKKLWGDRYFDPANGKFSKSATSPEGKKLPRTFCQLILDPIFKVFDAIMNFKKEETAKLIEKLDIKLDSEDKDKEGKPLLKAVMRRWLPAGDALLQMITIHLPSPVTAQKYRCELLYEGPPDDEAAMGIKSCDPKGPLMMYISKMVPTSDKGRFYAFGRVFSGLVSTGLKVRIMGPNYTPGKKEDLYLKPIQRTILMMGRYVEPIEDVPCGNIVGLVGVDQFLVKTGTITTFEHAHNMRVMKFSVSPVVRVAVEAKNPADLPKLVEGLKRLAKSDPMVQCIIEESGEHIIAGAGELHLEICLKDLEEDHACIPIKKSDPVVSYRETVSEESNVLCLSKSPNKHNRLYMKARPFPDGLAEDIDKGEVSARQELKQRARYLAEKYEWDVAEARKIWCFGPDGTGPNILTDITKGVQYLNEIKDSVVAGFQWATKEGALCEENMRGVRFDVHDVTLHADAIHRGGGQIIPTARRCLYASVLTAQPRLMEPIYLVEIQCPEQVVGGIYGVLNRKRGHVFEESQVAGTPMFVVKAYLPVNESFGFTADLRSNTGGQAFPQCVFDHWQILPGDPFDNSSRPSQVVAETRKRKGLKEGIPALDNFLDKL",
# PLST_HUMAN
"MDEMATTQISKDELDELKEAFAKVDLNSNGFICDYELHELFKEANMPLPGYKVREIIQKLMLDGDRNKDGKISFDEFVYIFQEVKSSDIAKTFRKAINRKEGICALGGTSELSSEGTQHSYSEEEKYAFVNWINKALENDPDCRHVIPMNPNTDDLFKAVGDGIVLCKMINLSVPDTIDERAINKKKLTPFIIQENLNLALNSASAIGCHVVNIGAEDLRAGKPHLVLGLLWQIIKIGLFADIELSRNEALAALLRDGETLEELMKLSPEELLLRWANFHLENSGWQKINNFSADIKDSKAYFHLLNQIAPKGQKEGEPRIDINMSGFNETDDLKRAESMLQQADKLGCRQFVTPADVVSGNPKLNLAFVANLFNKYPALTKPENQDIDWTLLEGETREERTFRNWMNSLGVNPHVNHLYADLQDALVILQLYERIKVPVDWSKVNKPPYPKLGANMKKLENCNYAVELGKHPAKFSLVGIGGQDLNDGNQTLTLALVWQLMRRYTLNVLEDLGDGQKANDDIIVNWVNRTLSEAGKSTSIQSFKDKTISSSLAVVDLIDAIQPGCINYDLVKSGNLTEDDKHNNAKYAVSMARRIGARVYALPEDLVEVKPKMVMTVFACLMGRGMKRV",
# KAP2_HUMAN
"MSHIQIPPGLTELLQGYTVEVLRQQPPDLVEFAVEYFTRLREARAPASVLPAATPRQSLGHPPPEPGPDRVADAKGDSESEEDEDLEVPVPSRFNRRVSVCAETYNPDEEEEDTDPRVIHPKTDEQRCRLQEACKDILLFKNLDQEQLSQVLDAMFERIVKADEHVIDQGDDGDNFYVIERGTYDILVTKDNQTRSVGQYDNRGSFGELALMYNTPRAATIVATSEGSLWGLDRVTFRRIIVKNNAKKRKMFESFIESVPLLKSLEVSERMKIVDVIGEKIYKDGERIITQGEKADSFYIIESGEVSILIRSRTKSNKDGGNQEVEIARCHKGQYFGELALVTNKPRAASAYAVGDVKCLVMDVQAFERLLGPCMDIMKRNISHYEEQLVKMFGSSVDLGNLGQ",
# DRD2_HUMAN
"MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIAVIVFGNVLVCMAVSREKALQTTTNYLIVSLAVADLLVATLVMPWVVYLEVVGEWKFSRIHCDIFVTLDVMMCTASILNLCAISIDRYTAVAMPMLYNTRYSSKRRVTVMISIVWVLSFTISCPLLFGLNNADQNECIIANPAFVVYSSIVSFYVPFIVTLLVYIKIYIVLRRRRKRVNTKRSSRAFRAHLRAPLKGNCTHPEDMKLCTVIMKSNGSFPVNRRRVEAARRAQELEMEMLSSTSPPERTRYSPIPPSHHQLTLPDPSHHGLHSTPDSPAKPEKNGHAKDHPKIAKIFEIQTMPNGKTRTSLKTMSRRKLSQQKEKKATQMLAIVLGVFIICWLPFFITHILNIHCDCNIPPVLYSAFTWLGYVNSAVNPIIYTTFNIEFRKAFLKILHC",
# AK1A1_HUMAN
"MAASCVLLHTGQKMPLIGLGTWKSEPGQVKAAVKYALSVGYRHIDCAAIYGNEPEIGEALKEDVGPGKAVPREELFVTSKLWNTKHHPEDVEPALRKTLADLQLEYLDLYLMHWPYAFERGDNPFPKNADGTICYDSTHYKETWKALEALVAKGLVQALGLSNFNSRQIDDILSVASVRPAVLQVECHPYLAQNELIAHCQARGLEVTAYSPLGSSDRAWRDPDEPVLLEEPVVLALAEKYGRSPAQILLRWQVQRKVICIPKSITPSRILQNIKVFDFTFSPEEMKQLNALNKNWRYIVPMLTVDGKRVPRDAGHPLYPFNDPY",
# GLUT4_HUMAN
"MPSGFQQIGSEDGEPPQQRVTGTLVLAVFSAVLGSLQFGYNIGVINAPQKVIEQSYNETWLGRQGPEGPSSIPPGTLTTLWALSVAIFSVGGMISSFLIGIISQWLGRKRAMLVNNVLAVLGGSLMGLANAAASYEMLILGRFLIGAYSGLTSGLVPMYVGEIAPTHLRGALGTLNQLAIVIGILIAQVLGLESLLGTASLWPLLLGLTVLPALLQLVLLPFCPESPRYLYIIQNLEGPARKSLKRLTGWADVSGVLAELKDEKRKLERERPLSLLQLLGSRTHRQPLIIAVVLQLSQQLSGINAVFYYSTSIFETAGVGQPAYATIGAGVVNTVFTLVSVLLVERAGRRTLHLLGLAGMCGCAILMTVALLLLERVPAMSYVSIVAIFGFVAFFEIGPGPIPWFIVAELFSQGPRPAAMAVAGFSNWTSNFIIGMGFQYVAEAMGPYVFLLFAVLLLGFFIFTFLRVPETRGRTFDQISAAFHRTPSLLEQEVKPSTELEYLGPDEND",
# MMP9_HUMAN
"MSLWQPLVLVLLVLGCCFAAPRQRQSTLVLFPGDLRTNLTDRQLAEEYLYRYGYTRVAEMRGESKSLGPALLLLQKQLSLPETGELDSATLKAMRTPRCGVPDLGRFQTFEGDLKWHHHNITYWIQNYSEDLPRAVIDDAFARAFALWSAVTPLTFTRVYSRDADIVIQFGVAEHGDGYPFDGKDGLLAHAFPPGPGIQGDAHFDDDELWSLGKGVVVPTRFGNADGAACHFPFIFEGRSYSACTTDGRSDGLPWCSTTANYDTDDRFGFCPSERLYTQDGNADGKPCQFPFIFQGQSYSACTTDGRSDGYRWCATTANYDRDKLFGFCPTRADSTVMGGNSAGELCVFPFTFLGKEYSTCTSEGRGDGRLWCATTSNFDSDKKWGFCPDQGYSLFLVAAHEFGHALGLDHSSVPEALMYPMYRFTEGPPLHKDDVNGIRHLYGPRPEPEPRPPTTTTPQPTAPPTVCPTGPPTVHPSERPTAGPTGPPSAGPTGPPTAGPSTATTVPLSPVDDACNVNIFDAIAEIGNQLYLFKDGKYWRFSEGRGSRPQGPFLIADKWPALPRKLDSVFEERLSKKLFFFSGRQVWVYTGASVLGPRRLDKLGLGADVAQVTGALRSGRGKMLLFSGRRLWRFDVKAQMVDPRSASEVDRMFPGVPLDTHDVFQYREKAYFCQDRFYWRVSSRSELNQVDQVGYVTYDILQCPED",
# HNRPL_HUMAN
"MSRRLLPRAEKRRRRLEQRQQPDEQRRRSGAMVKMAAAGGGGGGGRYYGGGSEGGRAPKRLKTDNAGDQHGGGGGGGGGAGAAGGGGGGENYDDPHKTPASPVVHIRGLIDGVVEADLVEALQEFGPISYVVVMPKKRQALVEFEDVLGACNAVNYAADNQIYIAGHPAFVNYSTSQKISRPGDSDDSRSVNSVLLFTILNPIYSITTDVLYTICNPCGPVQRIVIFRKNGVQAMVEFDSVQSAQRAKASLNGADIYSGCCTLKIEYAKPTRLNVFKNDQDTWDYTNPNLSGQGDPGSNPNKRQRQPPLLGDHPAEYGGPHGGYHSHYHDEGYGPPPPHYEGRRMGPPVGGHRRGPSRYGPQYGHPPPPPPPPEYGPHADSPVLMVYGLDQSKMNCDRVFNVFCLYGNVEKVKFMKSKPGAAMVEMADGYAVDRAITHLNNNFMFGQKLNVCVSKQPAIMPGQSYGLEDGSCSYKDFSESRNNRFSTPEQAAKNRIQHPSNVLHFFNAPLEVTEENFFEICDELGVKRPSSVKVFSGKSERSSSGLLEWESKSDALETLGFLNHYQMKNPNGPYPYTLKLCFSTAQHAS",
# GBRA1_HUMAN
"MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRILDRLLDGYDNRLRPGLGERVTEVKTDIFVTSFGPVSDHDMEYTIDVFFRQSWKDERLKFKGPMTVLRLNNLMASKIWTPDTFFHNGKKSVAHNMTMPNKLLRITEDGTLLYTMRLTVRAECPMHLEDFPMDAHACPLKFGSYAYTRAEVVYEWTREPARSVVVAEDGSRLNQYDLLGQTVDSGIVQSSTGEYVVMTTHFHLKRKIGYFVIQTYLPCIMTVILSQVSFWLNRESVPARTVFGVTTVLTMTTLSISARNSLPKVAYATAMDWFIAVCYAFVFSALIEFATVNYFTKRGYAWDGKSVVPEKPKKVKDPLIKKNNTYAPTATSYTPNLARGDPGLATIAKSATIEPKEVKPETKPPEPKKTFNSVSKIDRLSRIAFPLLFGIFNLVYWATYLNREPQLKAPTPHQ",
# BRAF_HUMAN
"MAALSGGGGGGAEPGQALFNGDMEPEAGAGAGAAASSAADPAIPEEVWNIKQMIKLTQEHIEALLDKFGGEHNPPSIYLEAYEEYTSKLDALQQREQQLLESLGNGTDFSVSSSASMDTVTSSSSSSLSVLPSSLSVFQNPTDVARSNPKSPQKPIVRVFLPNKQRTVVPARCGVTVRDSLKKALMMRGLIPECCAVYRIQDGEKKPIGWDTDISWLTGEELHVEVLENVPLTTHNFVRKTFFTLAFCDFCRKLLFQGFRCQTCGYKFHQRCSTEVPLMCVNYDQLDLLFVSKFFEHHPIPQEEASLAETALTSGSSPSAPASDSIGPQILTSPSPSKSIPIPQPFRPADEDHRNQFGQRDRSSSAPNVHINTIEPVNIDDLIRDQGFRGDGGSTTGLSATPPASLPGSLTNVKALQKSPGPQRERKSSSSSEDRNRMKTLGRRDSSDDWEIPDGQITVGQRIGSGSFGTVYKGKWHGDVAVKMLNVTAPTPQQLQAFKNEVGVLRKTRHVNILLFMGYSTKPQLAIVTQWCEGSSLYHHLHIIETKFEMIKLIDIARQTAQGMDYLHAKSIIHRDLKSNNIFLHEDLTVKIGDFGLATVKSRWSGSHQFEQLSGSILWMAPEVIRMQDKNPYSFQSDVYAFGIVLYELMTGQLPYSNINNRDQIIFMVGRGYLSPDLSKVRSNCPKAMKRLMAECLKKKRDERPLFPQILASIELLARSLPKIHRSASEPSLNRAGFQTEDFSLYACASPKTPIQAGGYGAFPVH",
# EZRI_HUMAN
"MPKPINVRVTTMDAELEFAIQPNTTGKQLFDQVVKTIGLREVWYFGLHYVDNKGFPTWLKLDKKVSAQEVRKENPLQFKFRAKFYPEDVAEELIQDITQKLFFLQVKEGILSDEIYCPPETAVLLGSYAVQAKFGDYNKEVHKSGYLSSERLIPQRVMDQHKLTRDQWEDRIQVWHAEHRGMLKDNAMLEYLKIAQDLEMYGINYFEIKNKKGTDLWLGVDALGLNIYEKDDKLTPKIGFPWSEIRNISFNDKKFVIKPIDKKAPDFVFYAPRLRINKRILQLCMGNHELYMRRRKPDTIEVQQMKAQAREEKHQKQLERQQLETEKKRRETVEREKEQMMREKEELMLRLQDYEEKTKKAERELSEQIQRALQLEEERKRAQEEAERLEADRMAALRAKEELERQAVDQIKSQEQLAAELAEYTAKIALLEEARRRKEDEVEEWQHRAKEAQDDLVKTKEELHLVMTAPPPPPPPVYEPVSYHVQESLQDEGAEPTGYSAELSSEGIRDDRNEEKRITEAEKNERVQRQLLTLSSELSQARDENKRTHNDIIHNENMRQGRDKYKTLRQIRQGNTKQRIDEFEAL",
# VATB1_HUMAN
"MAMEIDSRPGGLPGSSCNLGAAREHMQAVTRNYITHPRVTYRTVCSVNGPLVVLDRVKFAQYAEIVHFTLPDGTQRSGQVLEVAGTKAIVQVFEGTSGIDARKTTCEFTGDILRTPVSEDMLGRVFNGSGKPIDKGPVVMAEDFLDINGQPINPHSRIYPEEMIQTGISPIDVMNSIARGQKIPIFSAAGLPHNEIAAQICRQAGLVKKSKAVLDYHDDNFAIVFAAMGVNMETARFFKSDFEQNGTMGNVCLFLNLANDPTIERIITPRLALTTAEFLAYQCEKHVLVILTDMSSYAEALREVSAAREEVPGRRGFPGYMYTDLATIYERAGRVEGRGGSITQIPILTMPNDDITHPIPDLTGFITEGQIYVDRQLHNRQIYPPINVLPSLSRLMKSAIGEGMTRKDHGDVSNQLYACYAIGKDVQAMKAVVGEEALTSEDLLYLEFLQKFEKNFINQGPYENRSVFESLDLGWKLLRIFPKEMLKRIPQAVIDEFYSREGALQDLAPDTAL",
# RS2_HUMAN
"MADDAGAAGGPGGPGGPGMGNRGGFRGGFGSGIRGRGRGRGRGRGRGRGARGGKAEDKEWMPVTKLGRLVKDMKIKSLEEIYLFSLPIKESEIIDFFLGASLKDEVLKIMPVQKQTRAGQRTRFKAFVAIGDYNGHVGLGVKCSKEVATAIRGAIILAKLSIVPVRRGYWGNKIGKPHTVPCKVTGRCGSVLVRLIPAPRGTGIVSAPVPKKLLMMAGIDDCYTSARGCTATLGNFAKATFDAISKTYSYLTPDLWKETVFTKSPYQEFTDHLVKTHTRVSVQRTQAPAVATT",
# CD44_HUMAN
"MDKFWWHAAWGLCLVPLSLAQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYPSNPTDDDVSSGSSSERSSTSGGYIFYTFSTVHPIPDEDSPWITDSTDRIPATTLMSTSATATETATKRQETWDWFSWLFLPSESKNHLHTTTQMAGTSSNTISAGWEPNEENEDERDRHLSFSGSGIDDDEDFISSTISTTPRAFDHTKQNQDWTQWNPSHSNPEVLLQTTTRMTDVDRNGTTAYEGNWNPEAHPPLIHHEHHEEEETPHSTSTIQATPSSTTEETATQKEQWFGNRWHEGYRQTPKEDSHSTTGTAAASAHTSHPMQGRTTPSPEDSSWTDFFNPISHPMGRGHQAGRRMDMDSSHSITLQPTANPNTGLVEDLDRTGPLSMTTQQSNSQSFSTSHEGLEEDKDHPTTSTLTSSNRNDVTGGRRDPNHSEGSTTLLEGYTSHYPHTKESRTFIPVTSAKTGSFGVTAVTVGDSNSNVNRSLSGDQDTFHPSGGSHTTHGSESDGHSHGSQEGGANTTSGPIRTPQIPEWLIILASLLALALILAVCIAVNSRRRCGQKKKLVINSGNGAVEDRKPSGLNGEASKSQEMVHLVNKESSETPDQFMTADETRNLQNVDMKIGV",
# PGCA_HUMAN
"MTTLLWVFVTLRVITAAVTVETSDHDNSLSVSIPQPSPLRVLLGTSLTIPCYFIDPMHPVTTAPSTAPLAPRIKWSRVSKEKEVVLLVATEGRVRVNSAYQDKVSLPNYPAIPSDATLEVQSLRSNDSGVYRCEVMHGIEDSEATLEVVVKGIVFHYRAISTRYTLDFDRAQRACLQNSAIIATPEQLQAAYEDGFHQCDAGWLADQTVRYPIHTPREGCYGDKDEFPGVRTYGIRDTNETYDVYCFAEEMEGEVFYATSPEKFTFQEAANECRRLGARLATTGQLYLAWQAGMDMCSAGWLADRSVRYPISKARPNCGGNLLGVRTVYVHANQTGYPDPSSRYDAICYTGEDFVDIPENFFGVGGEEDITVQTVTWPDMELPLPRNITEGEARGSVILTVKPIFEVSPSPLEPEEPFTFAPEIGATAFAEVENETGEATRPWGFPTPGLGPATAFTSEDLVVQVTAVPGQPHLPGGVVFHYRPGPTRYSLTFEEAQQACLRTGAVIASPEQLQAAYEAGYEQCDAGWLRDQTVRYPIVSPRTPCVGDKDSSPGVRTYGVRPSTETYDVYCFVDRLEGEVFFATRLEQFTFQEALEFCESHNATLATTGQLYAAWSRGLDKCYAGWLADGSLRYPIVTPRPACGGDKPGVRTVYLYPNQTGLPDPLSRHHAFCFRGISAVPSPGEEEGGTPTSPSGVEEWIVTQVVPGVAAVPVEEETTAVPSGETTAILEFTTEPENQTEWEPAYTPVGTSPLPGILPTWPPTGAATEESTEGPSATEVPSASEEPSPSEVPFPSEEPSPSEEPFPSVRPFPSVELFPSEEPFPSKEPSPSEEPSASEEPYTPSPPVPSWTELPSSGEESGAPDVSGDFTGSGDVSGHLDFSGQLSGDRASGLPSGDLDSSGLTSTVGSGLPVESGLPSGDEERIEWPSTPTVGELPSGAEILEGSASGVGDLSGLPSGEVLETSASGVGDLSGLPSGEVLETTAPGVEDISGLPSGEVLETTAPGVEDISGLPSGEVLETTAPGVEDISGLPSGEVLETTAPGVEDISGLPSGEVLETTAPGVEDISGLPSGEVLETTAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETAAPGVEDISGLPSGEVLETTAPGVEEISGLPSGEVLETTAPGVDEISGLPSGEVLETTAPGVEEISGLPSGEVLETSTSAVGDLSGLPSGGEVLEISVSGVEDISGLPSGEVVETSASGIEDVSELPSGEGLETSASGVEDLSRLPSGEEVLEISASGFGDLSGLPSGGEGLETSASEVGTDLSGLPSGREGLETSASGAEDLSGLPSGKEDLVGSASGDLDLGKLPSGTLGSGQAPETSGLPSGFSGEYSGVDLGSGPPSGLPDFSGLPSGFPTVSLVDSTLVEVVTASTASELEGRGTIGISGAGEISGLPSSELDISGRASGLPSGTELSGQASGSPDVSGEIPGLFGVSGQPSGFPDTSGETSGVTELSGLSSGQPGISGEASGVLYGTSQPFGITDLSGETSGVPDLSGQPSGLPGFSGATSGVPDLVSGTTSGSGESSGITFVDTSLVEVAPTTFKEEEGLGSVELSGLPSGEADLSGKSGMVDVSGQFSGTVDSSGFTSQTPEFSGLPSGIAEVSGESSRAEIGSSLPSGAYYGSGTPSSFPTVSLVDRTLVESVTQAPTAQEAGEGPSGILELSGAHSGAPDMSGEHSGFLDLSGLQSGLIEPSGEPPGTPYFSGDFASTTNVSGESSVAMGTSGEASGLPEVTLITSEFVEGVTEPTISQELGQRPPVTHTPQLFESSGKVSTAGDISGATPVLPGSGVEVSSVPESSSETSAYPEAGFGASAAPEASREDSGSPDLSETTSAFHEANLERSSGLGVSGSTLTFQEGEASAAPEVSGESTTTSDVGTEAPGLPSATPTASGDRTEISGDLSGHTSQLGVVISTSIPESEWTQQTQRPAETHLEIESSSLLYSGEETHTVETATSPTDASIPASPEWKRESESTAAAPARSCAEEPCGAGTCKETEGHVICLCPPGYTGEHCNIDQEVCEEGWNKYQGHCYRHFPDRETWVDAERRCREQQSHLSSIVTPEEQEFVNNNAQDYQWIGLNDRTIEGDFRWSDGHPMQFENWRPNQPDNFFAAGEDCVVMIWHEKGEWNDVPCNYHLPFTCKKGTVACGEPPVVEHARTFGQKKDRYEINSLVRYQCTEGFVQRHMPTIRCQPSGHWEEPQITCTDPTTYKRRLQKRSSRHPRRSRPSTAH",
# ITB4_HUMAN
"MAGPRPSPWARLLLAALISVSLSGTLANRCKKAPVKSCTECVRVDKDCAYCTDEMFRDRRCNTQAELLAAGCQRESIVVMESSFQITEETQIDTTLRRSQMSPQGLRVRLRPGEERHFELEVFEPLESPVDLYILMDFSNSMSDDLDNLKKMGQNLARVLSQLTSDYTIGFGKFVDKVSVPQTDMRPEKLKEPWPNSDPPFSFKNVISLTEDVDEFRNKLQGERISGNLDAPEGGFDAILQTAVCTRDIGWRPDSTHLLVFSTESAFHYEADGANVLAGIMSRNDERCHLDTTGTYTQYRTQDYPSVPTLVRLLAKHNIIPIFAVTNYSYSYYEKLHTYFPVSSLGVLQEDSSNIVELLEEAFNRIRSNLDIRALDSPRGLRTEVTSKMFQKTRTGSFHIRRGEVGIYQVQLRALEHVDGTHVCQLPEDQKGNIHLKPSFSDGLKMDAGIICDVCTCELQKEVRSARCSFNGDFVCGQCVCSEGWSGQTCNCSTGSLSDIQPCLREGEDKPCSGRGECQCGHCVCYGEGRYEGQFCEYDNFQCPRTSGFLCNDRGRCSMGQCVCEPGWTGPSCDCPLSNATCIDSNGGICNGRGHCECGRCHCHQQSLYTDTICEINYSAIHPGLCEDLRSCVQCQAWGTGEKKGRTCEECNFKVKMVDELKRAEEVVVRCSFRDEDDDCTYSYTMEGDGAPGPNSTVLVHKKKDCPPGSFWWLIPLLLLLLPLLALLLLLCWKYCACCKACLALLPCCNRGHMVGFKEDHYMLRENLMASDHLDTPMLRSGNLKGRDVVRWKVTNNMQRPGFATHAASINPTELVPYGLSLRLARLCTENLLKPDTRECAQLRQEVEENLNEVYRQISGVHKLQQTKFRQQPNAGKKQDHTIVDTVLMAPRSAKPALLKLTEKQVEQRAFHDLKVAPGYYTLTADQDARGMVEFQEGVELVDVRVPLFIRPEDDDEKQLLVEAIDVPAGTATLGRRLVNITIIKEQARDVVSFEQPEFSVSRGDQVARIPVIRRVLDGGKSQVSYRTQDGTAQGNRDYIPVEGELLFQPGEAWKELQVKLLELQEVDSLLRGRQVRRFHVQLSNPKFGAHLGQPHSTTIIIRDPDELDRSFTSQMLSSQPPPHGDLGAPQNPNAKAAGSRKIHFNWLPPSGKPMGYRVKYWIQGDSESEAHLLDSKVPSVELTNLYPYCDYEMKVCAYGAQGEGPYSSLVSCRTHQEVPSEPGRLAFNVVSSTVTQLSWAEPAETNGEITAYEVCYGLVNDDNRPIGPMKKVLVDNPKNRMLLIENLRESQPYRYTVKARNGAGWGPEREAIINLATQPKRPMSIPIIPDIPIVDAQSGEDYDSFLMYSDDVLRSPSGSQRPSVSDDTGCGWKFEPLLGEELDLRRVTWRLPPELIPRLSASSGRSSDAEAPHGPPDDGGAGGKGGSLPRSATPGPPGEHLVNGRMDFAFPGSTNSLHRMTTTSAAAYGTHLSPHVPHRVLSTSSTLTRDYNSLTRSEHSHSTTLPRDYSTLTSVSSHDSRLTAGVPDTPTRLVFSALGPTSLRVSWQEPRCERPLQGYSVEYQLLNGGELHRLNIPNPAQTSVVVEDLLPNHSYVFRVRAQSQEGWGREREGVITIESQVHPQSPLCPLPGSAFTLSTPSAPGPLVFTALSPDSLQLSWERPRRPNGDIVGYLVTCEMAQGGGPATAFRVDGDSPESRLTVPGLSENVPYKFKVQARTTEGFGPEREGIITIESQDGGPFPQLGSRAGLFQHPLQSEYSSITTTHTSATEPFLVDGLTLGAQHLEAGGSLTRHVTQEFVSRTLTTSGTLSTHMDQQFFQT",
# GDC_HUMAN
"MAAATAAAALAAADPPPAMPQAAGAGGPTTRRDFYWLRSFLAGGIAGCCAKTTVAPLDRVKVLLQAHNHHYKHLGVFSALRAVPQKEGFLGLYKGNGAMMIRIFPYGAIQFMAFEHYKTLITTKLGISGHVHRLMAGSMAGMTAVICTYPLDMVRVRLAFQVKGEHSYTGIIHAFKTIYAKEGGFFGFYRGLMPTILGMAPYAGVSFFTFGTLKSVGLSHAPTLLGRPSSDNPNVLVLKTHVNLLCGGVAGAIAQTISYPFDVTRRRMQLGTVLPEFEKCLTMRDTMKYVYGHHGIRKGLYRGLSLNYIRCIPSQAVAFTTYELMKQFFHLN",
# PP2BB_HUMAN
"MAAPEPARAAPPPPPPPPPPPGADRVVKAVPFPPTHRLTSEEVFDLDGIPRVDVLKNHLVKEGRVDEEIALRIINEGAAILRREKTMIEVEAPITVCGDIHGQFFDLMKLFEVGGSPANTRYLFLGDYVDRGYFSIECVLYLWVLKILYPSTLFLLRGNHECRHLTEYFTFKQECKIKYSERVYEACMEAFDSLPLAALLNQQFLCVHGGLSPEIHTLDDIRRLDRFKEPPAFGPMCDLLWSDPSEDFGNEKSQEHFSHNTVRGCSYFYNYPAVCEFLQNNNLLSIIRAHEAQDAGYRMYRKSQTTGFPSLITIFSAPNYLDVYNNKAAVLKYENNVMNIRQFNCSPHPYWLPNFMDVFTWSLPFVGEKVTEMLVNVLSICSDDELMTEGEDQFDGSAAARKEIIRNKIRAIGKMARVFSVLREESESVLTLKGLTPTGMLPSGVLAGGRQTLQSATVEAIEAEKAIRGFSPPHRICSFEEAKGLDRINERMPPRKDAVQQDGFNSLNTAHATENHGTGNHTAQ",
# KCNA2_HUMAN
"MTVATGDPADEAAALPGHPQDTYDPEADHECCERVVINISGLRFETQLKTLAQFPETLLGDPKKRMRYFDPLRNEYFFDRNRPSFDAILYYYQSGGRLRRPVNVPLDIFSEEIRFYELGEEAMEMFREDEGYIKEEERPLPENEFQRQVWLLFEYPESSGPARIIAIVSVMVILISIVSFCLETLPIFRDENEDMHGSGVTFHTYSNSTIGYQQSTSFTDPFFIVETLCIIWFSFEFLVRFFACPSKAGFFTNIMNIIDIVAIIPYFITLGTELAEKPEDAQQGQQAMSLAILRVIRLVRVFRIFKLSRHSKGLQILGQTLKASMRELGLLIFFLFIGVILFSSAVYFAEADERESQFPSIPDAFWWAVVSMTTVGYGDMVPTTIGGKIVGSLCAIAGVLTIALPVPVIVSNFNYFYHRETEGEEQAQYLQVTSCPKIPSSPDLKKSRSASTISKSDYMEIQEGVNNSNEDFREENLKTANCTLANTNYVNITKMLTDV",
# YBOX3_HUMAN
"MSEAGEATTTTTTTLPQAPTEAAAAAPQDPAPKSPVGSGAPQAAAPAPAAHVAGNPGGDAAPAATGTAAAASLATAAGSEDAEKKVLATKVLGTVKWFNVRNGYGFINRNDTKEDVFVHQTAIKKNNPRKYLRSVGDGETVEFDVVEGEKGAEAANVTGPDGVPVEGSRYAADRRRYRRGYYGRRRGPPRNYAGEEEEEGSGSSEGFDPPATDRQFSGARNQLRRPQYRPQYRQRRFPPYHVGQTFDRRSRVLPHPNRIQAGEIGEMKDGVPEGAQLQGPVHRNPTYRPRYRSRGPPRPRPAPAVGEAEDKENQQATSGPNQPSVRRGYRRPYNYRRRPRPPNAPSQDGKEAKAGEAPTENPAPPTQQSSAE",
# KPCA_HUMAN
"MADVFPGNDSTASQDVANRFARKGALRQKNVHEVKDHKFIARFFKQPTFCSHCTDFIWGFGKQGFQCQVCCFVVHKRCHEFVTFSCPGADKGPDTDDPRSKHKFKIHTYGSPTFCDHCGSLLYGLIHQGMKCDTCDMNVHKQCVINVPSLCGMDHTEKRGRIYLKAEVADEKLHVTVRDAKNLIPMDPNGLSDPYVKLKLIPDPKNESKQKTKTIRSTLNPQWNESFTFKLKPSDKDRRLSVEIWDWDRTTRNDFMGSLSFGVSELMKMPASGWYKLLNQEEGEYYNVPIPEGDEEGNMELRQKFEKAKLGPAGNKVISPSEDRKQPSNNLDRVKLTDFNFLMVLGKGSFGKVMLADRKGTEELYAIKILKKDVVIQDDDVECTMVEKRVLALLDKPPFLTQLHSCFQTVDRLYFVMEYVNGGDLMYHIQQVGKFKEPQAVFYAAEISIGLFFLHKRGIIYRDLKLDNVMLDSEGHIKIADFGMCKEHMMDGVTTRTFCGTPDYIAPEIIAYQPYGKSVDWWAYGVLLYEMLAGQPPFDGEDEDELFQSIMEHNVSYPKSLSKEAVSVCKGLMTKHPAKRLGCGPEGERDVREHAFFRRIDWEKLENREIQPPFKPKVCGKGAENFDKFFTRGQPVLTPPDQLVIANIDQSDFEGFSYVNPQFVHPILQSAV",
# ITA2_HUMAN
"MGPERTGAAPLPLLLVLALSQGILNCCLAYNVGLPEAKIFSGPSSEQFGYAVQQFINPKGNWLLVGSPWSGFPENRMGDVYKCPVDLSTATCEKLNLQTSTSIPNVTEMKTNMSLGLILTRNMGTGGFLTCGPLWAQQCGNQYYTTGVCSDISPDFQLSASFSPATQPCPSLIDVVVVCDESNSIYPWDAVKNFLEKFVQGLDIGPTKTQVGLIQYANNPRVVFNLNTYKTKEEMIVATSQTSQYGGDLTNTFGAIQYARKYAYSAASGGRRSATKVMVVVTDGESHDGSMLKAVIDQCNHDNILRFGIAVLGYLNRNALDTKNLIKEIKAIASIPTERYFFNVSDEAALLEKAGTLGEQIFSIEGTVQGGDNFQMEMSQVGFSADYSSQNDILMLGAVGAFGWSGTIVQKTSHGHLIFPKQAFDQILQDRNHSSYLGYSVAAISTGESTHFVAGAPRANYTGQIVLYSVNENGNITVIQAHRGDQIGSYFGSVLCSVDVDKDTITDVLLVGAPMYMSDLKKEEGRVYLFTIKEGILGQHQFLEGPEGIENTRFGSAIAALSDINMDGFNDVIVGSPLENQNSGAVYIYNGHQGTIRTKYSQKILGSDGAFRSHLQYFGRSLDGYGDLNGDSITDVSIGAFGQVVQLWSQSIADVAIEASFTPEKITLVNKNAQIILKLCFSAKFRPTKQNNQVAIVYNITLDADGFSSRVTSRGLFKENNERCLQKNMVVNQAQSCPEHIIYIQEPSDVVNSLDLRVDISLENPGTSPALEAYSETAKVFSIPFHKDCGEDGLCISDLVLDVRQIPAAQEQPFIVSNQNKRLTFSVTLKNKRESAYNTGIVVDFSENLFFASFSLPVDGTEVTCQVAASQKSVACDVGYPALKREQQVTFTINFDFNLQNLQNQASLSFQALSESQEENKADNLVNLKIPLLYDAEIHLTRSTNINFYEISSDGNVPSIVHSFEDVGPKFIFSLKVTTGSVPVSMATVIIHIPQYTKEKNPLMYLTGVQTDKAGDISCNADINPLKIGQTSSSVSFKSENFRHTKELNCRTASCSNVTCWLKDVHMKGEYFVNVTTRIWNGTFASSTFQTVQLTAAAEINTYNPEIYVIEDNTVTIPLMIMKPDEKAEVPTGVIIGSIIAGILLLLALVAILWKLGFFKRKYEKMTKNPDEIDETTELSS",
# SYN1_HUMAN
"MNYLRRRLSDSNFMANLPNGYMTDLQRPQPPPPPPGAHSPGATPGPGTATAERSSGVAPAASPAAPSPGSSGGGGFFSSLSNAVKQTTAAAAATFSEQVGGGSGGAGRGGAASRVLLVIDEPHTDWAKYFKGKKIHGEIDIKVEQAEFSDLNLVAHANGGFSVDMEVLRNGVKVVRSLKPDFVLIRQHAFSMARNGDYRSLVIGLQYAGIPSVNSLHSVYNFCDKPWVFAQMVRLHKKLGTEEFPLIDQTFYPNHKEMLSSTTYPVVVKMGHAHSGMGKVKVDNQHDFQDIASVVALTKTYATAEPFIDAKYDVRVQKIGQNYKAYMRTSVSGNWKTNTGSAMLEQIAMSDRYKLWVDTCSEIFGGLDICAVEALHGKDGRDHIIEVVGSSMPLIGDHQDEDKQLIVELVVNKMAQALPRQRQRDASPGRGSHGQTPSPGALPLGRQTSQQPAGPPAQQRPPPQGGPPQPGPGPQRQGPPLQQRPPPQGQQHLSGLGPPAGSPLPQRLPSPTSAPQQPASQAAPPTQGQGRQSRPVAGGPGAPPAARPPASPSPQRQAGPPQATRQTSVSGPAPPKASGAPPGGQQRQGPPQKPPGPAGPTRQASQAGPVPRTGPPTTQQPRPSGPGPAGRPKPQLAQKPSQDVPPPATAAAGGPPHPQLNKSQSLTNAFNLPEPAPPRPSLSQDEVKAETIRSLRKSFASLFSD",
# KAPCA_HUMAN
"MGNAAAAKKGSEQESVKEFLAKAKEDFLKKWESPAQNTAHLDQFERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQIEHTLNEKRILQAVNFPFLVKLEFSFKDNSNLYMVMEYVPGGEMFSHLRRIGRFSEPHARFYAAQIVLTFEYLHSLDLIYRDLKPENLLIDQQGYIQVTDFGFAKRVKGRTWTLCGTPEYLAPEIILSKGYNKAVDWWALGVLIYEMAAGYPPFFADQPIQIYEKIVSGKVRFPSHFSSDLKDLLRNLLQVDLTKRFGNLKNGVNDIKNHKWFATTDWIAIYQRKVEAPFIPKFKGPGDTSNFDDYEEEEIRVSINEKCGKEFSEF",
# CAN2_HUMAN
"MAGIAAKLAKDREAAEGLGSHDRAIKYLNQDYEALRNECLEAGTLFQDPSFPAIPSALGFKELGPYSSKTRGIEWKRPTEICADPQFIIGGATRTDICQGALGDCWLLAAIASLTLNEEILARVVPLNQSFQENYAGIFHFQFWQYGEWVEVVVDDRLPTKDGELLFVHSAEGSEFWSALLEKAYAKINGCYEALSGGATTEGFEDFTGGIAEWYELKKPPPNLFKIIQKALQKGSLLGCSIDITSAADSEAITFQKLVKGHAYSVTGAEEVESNGSLQKLIRIRNPWGEVEWTGRWNDNCPSWNTIDPEERERLTRRHEDGEFWMSFSDFLRHYSRLEICNLTPDTLTSDTYKKWKLTKMDGNWRRGSTAGGCRNYPNTFWMNPQYLIKLEEEDEDEEDGESGCTFLVGLIQKHRRRQRKMGEDMHTIGFGIYEVPEELSGQTNIHLSKNFFLTNRARERSDTFINLREVLNRFKLPPGEYILVPSTFEPNKDGDFCIRVFSEKKADYQAVDDEIEANLEEFDISEDDIDDGFRRLFAQLAGEDAEISAFELQTILRRVLAKRQDIKSDGFSIETCKIMVDMLDSDGSGKLGLKEFYILWTKIQKYQKIYREIDVDRSGTMNSYEMRKALEEAGFKMPCQLHQVIVARFADDQLIIDFDNFVRCLVRLETLFKIFKQLDPENTGTIELDLISWLCFSVL",
# NEUM_HUMAN
"MLCCMRRTKQVEKNDDDQKIEQDGIKPEDKAHKAATKIQASFRGHITRKKLKGEKKDDVQAAEAEANKKDEAPVADGVEKKGEGTTTAEAAPATGSKPDEPGKAGETPSEEKKGEGDAATEQAAPQAPASSEEKAGSAETESATKASTDNSPSSKAEDAPAKEEPKQADVPAAVTAAAATTPAAEDAAAKATAQPPTETGESSQAEENIEAVDETKPKESARQDEGKEEEPEADQEHA",
# ACHB2_HUMAN
"MARRCGPVALLLGFGLLRLCSGVWGTDTEERLVEHLLDPSRYNKLIRPATNGSELVTVQLMVSLAQLISVHEREQIMTTNVWLTQEWEDYRLTWKPEEFDNMKKVRLPSKHIWLPDVVLYNNADGMYEVSFYSNAVVSYDGSIFWLPPAIYKSACKIEVKHFPFDQQNCTMKFRSWTYDRTEIDLVLKSEVASLDDFTPSGEWDIVALPGRRNENPDDSTYVDITYDFIIRRKPLFYTINLIIPCVLITSLAILVFYLPSDCGEKMTLCISVLLALTVFLLLISKIVPPTSLDVPLVGKYLMFTMVLVTFSIVTSVCVLNVHHRSPTTHTMAPWVKVVFLEKLPALLFMQQPRHHCARQRLRLRRRQREREGAGALFFREAPGADSCTCFVNRASVQGLAGAFGAEPAPVAGPGRSGEPCGCGLREAVDGVRFIADHMRSEDDDQSVSEDWKYVAMVIDRLFLWIFVFVCVFGTIGMFLQPLFQNYTTTTFLHSDHSAPSSK",
# PTN1_HUMAN
"MEMEKEFEQIDKSGSWAAIYQDIRHEASDFPCRVAKLPKNKNRNRYRDVSPFDHSRIKLHQEDNDYINASLIKMEEAQRSYILTQGPLPNTCGHFWEMVWEQKSRGVVMLNRVMEKGSLKCAQYWPQKEEKEMIFEDTNLKLTLISEDIKSYYTVRQLELENLTTQETREILHFHYTTWPDFGVPESPASFLNFLFKVRESGSLSPEHGPVVVHCSAGIGRSGTFCLADTCLLLMDKRKDPSSVDIKKVLLEMRKFRMGLIQTADQLRFSYLAVIEGAKFIMGDSSVQDQWKELSHEDLEPPPEHIPPPPRPPKRILEPHNGKCREFFPNHQWVKEETQEDKDCPIKEEKGSPLNAAPYGIESMSQDTEVRSRVVGGSLRGAQAASPAKGEPSLPEKDEDHALSYWKPFLVNMCVATVLTAGAYLCYRFLFNSNT",
# RL35A_HUMAN
"MSGRLWSKAIFAGYKRGLRNQREHTALLKIEGVYARDETEFYLGKRCAYVYKAKNNTVTPGGKPNKTRVIWGKVTRAHGNSGMVRAKFRSNLPAKAIGHRIRVMLYPSRI",
# ITB5_HUMAN
"MPRAPAPLYACLLGLCALLPRLAGLNICTSGSATSCEECLLIHPKCAWCSKEDFGSPRSITSRCDLRANLVKNGCGGEIESPASSFHVLRSLPLSSKGSGSAGWDVIQMTPQEIAVNLRPGDKTTFQLQVRQVEDYPVDLYYLMDLSLSMKDDLDNIRSLGTKLAEEMRKLTSNFRLGFGSFVDKDISPFSYTAPRYQTNPCIGYKLFPNCVPSFGFRHLLPLTDRVDSFNEEVRKQRVSRNRDAPEGGFDAVLQAAVCKEKIGWRKDALHLLVFTTDDVPHIALDGKLGGLVQPHDGQCHLNEANEYTASNQMDYPSLALLGEKLAENNINLIFAVTKNHYMLYKNFTALIPGTTVEILDGDSKNIIQLIINAYNSIRSKVELSVWDQPEDLNLFFTATCQDGVSYPGQRKCEGLKIGDTASFEVSLEARSCPSRHTEHVFALRPVGFRDSLEVGVTYNCTCGCSVGLEPNSARCNGSGTYVCGLCECSPGYLGTRCECQDGENQSVYQNLCREAEGKPLCSGRGDCSCNQCSCFESEFGKIYGPFCECDNFSCARNKGVLCSGHGECHCGECKCHAGYIGDNCNCSTDISTCRGRDGQICSERGHCLCGQCQCTEPGAFGEMCEKCPTCPDACSTKRDCVECLLLHSGKPDNQTCHSLCRDEVITWVDTIVKDDQEAVLCFYKTAKDCVMMFTYVELPSGKSNLTVLREPECGNTPNAMTILLAVVGSILLVGLALLAIWKLLVTIHDRREFAKFQSERSRARYEMASNPLYRKPISTHTVDFTFNKFNKSYNGTVD",
# ARF4_HUMAN
"MGLTISSLFSRLFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTIPTIGFNVETVEYKNICFTVWDVGGQDRIRPLWKHYFQNTQGLIFVVDSNDRERIQEVADELQKMLLVDELRDAVLLLFANKQDLPNAMAISEMTDKLGLQSLRNRTWYVQATCATQGTGLYEGLDWLSNELSKR",
# RL7_HUMAN
"MEGVEEKKKEVPAVPETLKKKRRNFAELKIKRLRKKFAQKMLRKARRKLIYEKAKHYHKEYRQMYRTEIRMARMARKAGNFYVPAEPKLAFVIRIRGINGVSPKVRKVLQLLRLRQIFNGTFVKLNKASINMLRIVEPYIAWGYPNLKSVNELIYKRGYGKINKKRIALTDNALIARSLGKYGIICMEDLIHEIYTVGKRFKEANNFLWPFKLSSPRGGMKKKTTHFVEGGDAGNREDQINRLIRRMN",
# PTPRA_HUMAN
"MDSWFILVLLGSGLICVSANNATTVAPSVGITRLINSSTAEPVKEEAKTSNPTSSLTSLSVAPTFSPNITLGPTYLTTVNSSDSDNGTTRTASTNSIGITISPNGTWLPDNQFTDARTEPWEGNSSTAATTPETFPPSGNSDSKDRRDETPIIAVMVALSSLLVIVFIIIVLYMLRFKKYKQAGSHSNSFRLSNGRTEDVEPQSVPLLARSPSTNRKYPPLPVDKLEEEINRRMADDNKLFREEFNALPACPIQATCEAASKEENKEKNRYVNILPYDHSRVHLTPVEGVPDSDYINASFINGYQEKNKFIAAQGPKEETVNDFWRMIWEQNTATIVMVTNLKERKECKCAQYWPDQGCWTYGNIRVSVEDVTVLVDYTVRKFCIQQVGDMTNRKPQRLITQFHFTSWPDFGVPFTPIGMLKFLKKVKACNPQYAGAIVVHCSAGVGRTGTFVVIDAMLDMMHTERKVDVYGFVSRIRAQRCQMVQTDMQYVFIYQALLEHYLYGDTELEVTSLETHLQKIYNKIPGTSNNGLEEEFKKLTSIKIQNDKMRTGNLPANMKKNRVLQIIPYEFNRVIIPVKRGEENTDYVNASFIDGYRQKDSYIASQGPLLHTIEDFWRMIWEWKSCSIVMLTELEERGQEKCAQYWPSDGLVSYGDITVELKKEEECESYTVRDLLVTNTRENKSRQIRQFHFHGWPEVGIPSDGKGMISIIAAVQKQQQQSGNHPITVHCSAGAGRTGTFCALSTVLERVKAEGILDVFQTVKSLRLQRPHMVQTLEQYEFCYKVVQEYIDAFSDYANFK",
# GBRB1_HUMAN
"MWTVQNRESLGLLSFPVMITMVCCAHSTNEPSNMSYVKETVDRLLKGYDIRLRPDFGGPPVDVGMRIDVASIDMVSEVNMDYTLTMYFQQSWKDKRLSYSGIPLNLTLDNRVADQLWVPDTYFLNDKKSFVHGVTVKNRMIRLHPDGTVLYGLRITTTAACMMDLRRYPLDEQNCTLEIESYGYTTDDIEFYWNGGEGAVTGVNKIELPQFSIVDYKMVSKKVEFTTGAYPRLSLSFRLKRNIGYFILQTYMPSTLITILSWVSFWINYDASAARVALGITTVLTMTTISTHLRETLPKIPYVKAIDIYLMGCFVFVFLALLEYAFVNYIFFGKGPQKKGASKQDQSANEKNKLEMNKVQVDAHGNILLSTLEIRNETSGSEVLTSVSDPKATMYSYDSASIQYRKPLSSREAYGRALDRHGVPSKGRIRRRASQLKVKIPDLTDVNSIDKWSRMFFPITFSLFNVVYWLYYVH",
# GBRG2_HUMAN
"MSSPNIWSTGSSVYSTPVFSQKMTVWILLLLSLYPGFTSQKSDDDYEDYASNKTWVLTPKVPEGDVTVILNNLLEGYDNKLRPDIGVKPTLIHTDMYVNSIGPVNAINMEYTIDIFFAQTWYDRRLKFNSTIKVLRLNSNMVGKIWIPDTFFRNSKKADAHWITTPNRMLRIWNDGRVLYTLRLTIDAECQLQLHNFPMDEHSCPLEFSSYGYPREEIVYQWKRSSVEVGDTRSWRLYQFSFVGLRNTTEVVKTTSGDYVVMSVYFDLSRRMGYFTIQTYIPCTLIVVLSWVSFWINKDAVPARTSLGITTVLTMTTLSTIARKSLPKVSYVTAMDLFVSVCFIFVFSALVEYGTLHYFVSNRKPSKDKDKKKKNPLLRMFSFKAPTIDIRPRSATIQMNNATHLQERDEEYGYECLDGKDCASFFCCFEDCRTGAWRHGRIHIRIAKMDSYARIFFPTAFCLFNLVYWVSYLYL",
# PACA_HUMAN
"MTMCSGARLALLVYGIIMHSSVYSSPAAAGLRFPGIRPEEEAYGEDGNPLPDFDGSEPPGAGSPASAPRAAAAWYRPAGRRDVAHGILNEAYRKVLDQLSAGKHLQSLVARGVGGSLGGGAGDDAEPLSKRHSDGIFTDSYSRYRKQMAVKKYLAAVLGKRYKQRVKNKGRRIAYL",
# RL17_HUMAN
"MVRYSLDPENPTKSCKSRGSNLRVHFKNTRETAQAIKGMHIRKATKYLKDVTLQKQCVPFRRYNGGVGRCAQAKQWGWTQGRWPKKSAEFLLHMLKNAESNAELKGLDVDSLVIEHIQVNKAPKMRRRTYRAHGRINPYMSSPCHIEMILTEKEQIVPKPEEEVAQKKKISQKKLKKQKLMARE",
# ADA2C_HUMAN
"MASPALAAALAVAAAAGPNASGAGERGSGGVANASGASWGPPRGQYSAGAVAGLAAVVGFLIVFTVVGNVLVVIAVLTSRALRAPQNLFLVSLASADILVATLVMPFSLANELMAYWYFGQVWCGVYLALDVLFCTSSIVHLCAISLDRYWSVTQAVEYNLKRTPRRVKATIVAVWLISAVISFPPLVSLYRQPDGAAYPQCGLNDETWYILSSCIGSFFAPCLIMGLVYARIYRVAKLRTRTLSEKRAPVGPDGASPTTENGLGAAAGAGENGHCAPPPADVEPDESSAAAERRRRRGALRRGGRRRAGAEGGAGGADGQGAGPGAAESGALTASRSPGPGGRLSRASSRSVEFFLSRRRRARSSVCRRKVAQAREKRFTFVLAVVMGVFVLCWFPFFFSYSLYGICREACQVPGPLFKFFFWIGYCNSSLNPVIYTVFNQDFRRSFKHILFRRRRRGFRQ",
# CADH2_HUMAN
"MCRIAGALRTLLPLLAALLQASVEASGEIALCKTGFPEDVYSAVLSKDVHEGQPLLNVKFSNCNGKRKVQYESSEPADFKVDEDGMVYAVRSFPLSSEHAKFLIYAQDKETQEKWQVAVKLSLKPTLTEESVKESAEVEEIVFPRQFSKHSGHLQRQKRDWVIPPINLPENSRGPFPQELVRIRSDRDKNLSLRYSVTGPGADQPPTGIFIINPISGQLSVTKPLDREQIARFHLRAHAVDINGNQVENPIDIVINVIDMNDNRPEFLHQVWNGTVPEGSKPGTYVMTVTAIDADDPNALNGMLRYRIVSQAPSTPSPNMFTINNETGDIITVAAGLDREKVQQYTLIIQATDMEGNPTYGLSNTATAVITVTDVNDNPPEFTAMTFYGEVPENRVDIIVANLTVTDKDQPHTPAWNAVYRISGGDPTGRFAIQTDPNSNDGLVTVVKPIDFETNRMFVLTVAAENQVPLAKGIQHPPQSTATVSVTVIDVNENPYFAPNPKIIRQEEGLHAGTMLTTFTAQDPDRYMQQNIRYTKLSDPANWLKIDPVNGQITTIAVLDRESPNVKNNIYNATFLASDNGIPPMSGTGTLQIYLLDINDNAPQVLPQEAETCETPDPNSINITALDYDIDPNAGPFAFDLPLSPVTIKRNWTITRLNGDFAQLNLKIKFLEAGIYEVPIIITDSGNPPKSNISILRVKVCQCDSNGDCTDVDRIVGAGLGTGAIIAILLCIIILLILVLMFVVWMKRRDKERQAKQLLIDPEDDVRDNILKYDEEGGGEEDQDYDLSQLQQPDTVEPDAIKPVGIRRMDERPIHAEPQYPVRSAAPHPGDIGDFINEGLKAADNDPTAPPYDSLLVFDYEGSGSTAGSLSSLNSSSSGGEQDYDYLNDWGPRFKKLADMYGGGDD",
# GNAZ_HUMAN
"MGCRQSSEEKEAARRSRRIDRHLRSESQRQRREIKLLLLGTSNSGKSTIVKQMKIIHSGGFNLEACKEYKPLIIYNAIDSLTRIIRALAALRIDFHNPDRAYDAVQLFALTGPAESKGEITPELLGVMRRLWADPGAQACFSRSSEYHLEDNAAYYLNDLERIAAADYIPTVEDILRSRDMTTGIVENKFTFKELTFKMVDVGGQRSERKKWIHCFEGVTAIIFCVELSGYDLKLYEDNQTSRMAESLRLFDSICNNNWFINTSLILFLNKKDLLAEKIRRIPLTICFPEYKGQNTYEEAAVYIQRQFEDLNRNKETKEIYSHFTCATDTSNIQFVFDAVTDVIIQNNLKYIGLC",
# PLCG1_HUMAN
"MAGAASPCANGCGPGAPSDAEVLHLCRSLEVGTVMTLFYSKKSQRPERKTFQVKLETRQITWSRGADKIEGAIDIREIKEIRPGKTSRDFDRYQEDPAFRPDQSHCFVILYGMEFRLKTLSLQATSEDEVNMWIKGLTWLMEDTLQAPTPLQIERWLRKQFYSVDRNREDRISAKDLKNMLSQVNYRVPNMRFLRERLTDLEQRSGDITYGQFAQLYRSLMYSAQKTMDLPFLEASTLRAGERPELCRVSLPEFQQFLLDYQGELWAVDRLQVQEFMLSFLRDPLREIEEPYFFLDEFVTFLFSKENSVWNSQLDAVCPDTMNNPLSHYWISSSHNTYLTGDQFSSESSLEAYARCLRMGCRCIELDCWDGPDGMPVIYHGHTLTTKIKFSDVLHTIKEHAFVASEYPVILSIEDHCSIAQQRNMAQYFKKVLGDTLLTKPVEISADGLPSPNQLKRKILIKHKKLAEGSAYEEVPTSMMYSENDISNSIKNGILYLEDPVNHEWYPHYFVLTSSKIYYSEETSSDQGNEDEEEPKEVSSSTELHSNEKWFHGKLGAGRDGRHIAERLLTEYCIETGAPDGSFLVRESETFVGDYTLSFWRNGKVQHCRIHSRQDAGTPKFFLTDNLVFDSLYDLITHYQQVPLRCNEFEMRLSEPVPQTNAHESKEWYHASLTRAQAEHMLMRVPRDGAFLVRKRNEPNSYAISFRAEGKIKHCRVQQEGQTVMLGNSEFDSLVDLISYYEKHPLYRKMKLRYPINEEALEKIGTAEPDYGALYEGRNPGFYVEANPMPTFKCAVKALFDYKAQREDELTFIKSAIIQNVEKQEGGWWRGDYGGKKQLWFPSNYVEEMVNPVALEPEREHLDENSPLGDLLRGVLDVPACQIAIRPEGKNNRLFVFSISMASVAHWSLDVAADSQEELQDWVKKIREVAQTADARLTEGKIMERRKKIALELSELVVYCRPVPFDEEKIGTERACYRDMSSFPETKAEKYVNKAKGKKFLQYNRLQLSRIYPKGQRLDSSNYDPLPMWICGSQLVALNFQTPDKPMQMNQALFMTGRHCGYVLQPSTMRDEAFDPFDKSSLRGLEPCAISIEVLGARHLPKNGRGIVCPFVEIEVAGAEYDSTKQKTEFVVDNGLNPVWPAKPFHFQISNPEFAFLRFVVYEEDMFSDQNFLAQATFPVKGLKTGYRAVPLKNNYSEDLELASLLIKIDIFPAKENGDLSPFSGTSLRERGSDASGQLFHGRAREGSFESRYQQPFEDFRISQEHLADHFDSRERRAPRRTRVNGDNRL",
# AT2B1_HUMAN
"MGDMANNSVAYSGVKNSLKEANHDGDFGITLAELRALMELRSTDALRKIQESYGDVYGICTKLKTSPNEGLSGNPADLERREAVFGKNFIPPKKPKTFLQLVWEALQDVTLIILEIAAIVSLGLSFYQPPEGDNALCGEVSVGEEEGEGETGWIEGAAILLSVVCVVLVTAFNDWSKEKQFRGLQSRIEQEQKFTVIRGGQVIQIPVADITVGDIAQVKYGDLLPADGILIQGNDLKIDESSLTGESDHVKKSLDKDPLLLSGTHVMEGSGRMVVTAVGVNSQTGIIFTLLGAGGEEEEKKDEKKKEKKNKKQDGAIENRNKAKAQDGAAMEMQPLKSEEGGDGDEKDKKKANLPKKEKSVLQGKLTKLAVQIGKAGLLMSAITVIILVLYFVIDTFWVQKRPWLAECTPIYIQYFVKFFIIGVTVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTMNRMTVVQAYINEKHYKKVPEPEAIPPNILSYLVTGISVNCAYTSKILPPEKEGGLPRHVGNKTECALLGLLLDLKRDYQDVRNEIPEEALYKVYTFNSVRKSMSTVLKNSDGSYRIFSKGASEIILKKCFKILSANGEAKVFRPRDRDDIVKTVIEPMASEGLRTICLAFRDFPAGEPEPEWDNENDIVTGLTCIAVVGIEDPVRPEVPDAIKKCQRAGITVRMVTGDNINTARAIATKCGILHPGEDFLCLEGKDFNRRIRNEKGEIEQERIDKIWPKLRVLARSSPTDKHTLVKGIIDSTVSDQRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFTSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTLASLALATEPPTESLLLRKPYGRNKPLISRTMMKNILGHAFYQLVVVFTLLFAGEKFFDIDSGRNAPLHAPPSEHYTIVFNTFVLMQLFNEINARKIHGERNVFEGIFNNAIFCTIVLGTFVVQIIIVQFGGKPFSCSELSIEQWLWSIFLGMGTLLWGQLISTIPTSRLKFLKEAGHGTQKEEIPEEELAEDVEEIDHAERELRRGQILWFRGLNRIQTQIRVVNAFRSSLYEGLEKPESRSSIHNFMTHPEFRIEDSEPHIPLIDDTDAEDDAPTKRNSSPPPSPNKNNNAVDSGIHLTIEMNKSATSSSPGSPLHSLETSL",
# IF2B_HUMAN
"MSGDEMIFDPTMSKKKKKKKKPFMLDEEGDTQTEETQPSETKEVEPEPTEDKDLEADEEDTRKKDASDDLDDLNFFNQKKKKKKTKKIFDIDEAEEGVKDLKIESDVQEPTEPEDDLDIMLGNKKKKKKNVKFPDEDEILEKDEALEDEDNKKDDGISFSNQTGPAWAGSERDYTYEELLNRVFNIMREKNPDMVAGEKRKFVMKPPQVVRVGTKKTSFVNFTDICKLLHRQPKHLLAFLLAELGTSGSIDGNNQLVIKGRFQQKQIENVLRRYIKEYVTCHTCRSPDTILQKDTRLYFLQCETCHSRCSVASIKTGFQAVTGKRAQLRAKAN",
# ACM3_HUMAN
"MTLHNNSTTSPLFPNISSSWIHSPSDAGLPPGTVTHFGSYNVSRAAGNFSSPDGTTDDPLGGHTVWQVVFIAFLTGILALVTIIGNILVIVSFKVNKQLKTVNNYFLLSLACADLIIGVISMNLFTTYIIMNRWALGNLACDLWLAIDYVASNASVMNLLVISFDRYFSITRPLTYRAKRTTKRAGVMIGLAWVISFVLWAPAILFWQYFVGKRTVPPGECFIQFLSEPTITFGTAIAAFYMPVTIMTILYWRIYKETEKRTKELAGLQASGTEAETENFVHPTGSSRSCSSYELQQQSMKRSNRRKYGRCHFWFTTKSWKPSSEQMDQDHSSSDSWNNNDAAASLENSASSDEEDIGSETRAIYSIVLKLPGHSTILNSTKLPSSDNLQVPEEELGMVDLERKADKLQAQKSVDDGGSFPKSFSKLPIQLESAVDTAKTSDVNSSVGKSTATLPLSFKEATLAKRFALKTRSQITKRKRMSLVKEKKAAQTLSAILLAFIITWTPYNIMVLVNTFCDSCIPKTFWNLGYWLCYINSTVNPVCYALCNKTFRTTFKMLLLCQCDKKKRRKQQYQQRQSVIFHKRAPEQAL",
# RAB3A_HUMAN
"MASATDSRYGQKESSDQNFDYMFKILIIGNSSVGKTSFLFRYADDSFTPAFVSTVGIDFKVKTIYRNDKRIKLQIWDTAGQERYRTITTAYYRGAMGFILMYDITNEESFNAVQDWSTQIKTYSWDNAQVLLVGNKCDMEDERVVSSERGRQLADHLGFEFFEASAKDNINVKQTFERLVDVICEKMSESLDTADPAVTGAKQGPQLSDQQVPPHQDCAC",
# RAB3B_HUMAN
"MASVTDGKTGVKDASDQNFDYMFKLLIIGNSSVGKTSFLFRYADDTFTPAFVSTVGIDFKVKTVYRHEKRVKLQIWDTAGQERYRTITTAYYRGAMGFILMYDITNEESFNAVQDWATQIKTYSWDNAQVILVGNKCDMEEERVVPTEKGQLLAEQLGFDFFEASAKENISVRQAFERLVDAICDKMSDSLDTDPSMLGSSKNTRLSDTPPLLQQNCSC",
# RAB4A_HUMAN
"MSQTAMSETYDFLFKFLVIGNAGTGKSCLLHQFIEKKFKDDSNHTIGVEFGSKIINVGGKYVKLQIWDTAGQERFRSVTRSYYRGAAGALLVYDITSRETYNALTNWLTDARMLASQNIVIILCGNKKDLDADREVTFLEASRFAQENELMFLETSALTGENVEEAFVQCARKILNKIESGELDPERMGSGIQYGDAALRQLRSPRRAQAPNAQECGC",
# RAB5A_HUMAN
"MASRGATRPNGPNTGNKICQFKLVLLGESAVGKSSLVLRFVKGQFHEFQESTIGAAFLTQTVCLDDTTVKFEIWDTAGQERYHSLAPMYYRGAQAAIVVYDITNEESFARAKNWVKELQRQASPNIVIALSGNKADLANKRAVDFQEAQSYADDNSLLFMETSAKTSMNVNEIFMAIAKKLPKNEPQNPGANSARGRGVDLTEPTQPTRNQCCSN",
# RAB6A_HUMAN
"MSTGGDFGNPLRKFKLVFLGEQSVGKTSLITRFMYDSFDNTYQATIGIDFLSKTMYLEDRTVRLQLWDTAGQERFRSLIPSYIRDSTVAVVVYDITNVNSFQQTTKWIDDVRTERGSDVIIMLVGNKTDLADKRQVSIEEGERKAKELNVMFIETSAKAGYNVKQLFRRVAAALPGMESTQDRSREDMIDIKLEKPQEQPVSEGGCSC",
# TKN1_HUMAN
"MKILVALAVFFLVSTQLFAEEIGANDDLNYWSDWYDSDQIKEELPEPFEHLLQRIARRPKPQQFFGLMGKRDADSSIEKQVALLKALYGHGQISHKRHKTDSFVGLMGKRALNSVAYERSAMQNYERRR",
# A1ATR_HUMAN
"MPFSVSWGILLLAGLCCLVPSSLVEDPQEDAAQKTDTSHHDQGDWEDLACQKISYNVTDLAFDLYKELADLSQTSNVLVTPTSVAMAFAMLSLGTKADTRTEILEGLNVNLTETPEAKIHECFQQVLQALSRPDTRLQLTTGSSLFVNKSMKLVDTFLEDTKKLYHSEASSINFRDTEEAKEQINNYVEKRTGRKVVDLVKHLKKDTSLALVDYISFHGKWKDKFKAEHIMVEGFHVDDKTIIRVPMINHLGRFDIHRDRELSSWVLAQHYVGNATAFFILPDPKKMWQLEEKLTYSHLENIQRAFDIRSINLHFPKLSISGTYKLKRVLRNLGITKIFSNEADLSGVSQEAPLKLSKAVHVAVLTIDEKGTEATGAPHLEEKAWSKYQTVMFNRPFLVIIKDDITNFPLFIGKVVNPTQK",
# PTN_HUMAN
"MQAQQYQQQRRKFAAAFLAFIFILAAVDTAEAGKKEKPEKKVKKSDCGEWQWSVCVPTSGDCGLGTREGTRTGAECKQTMKTQRCKIPCNWKKQFGAECKYQFQAWGECDLNTALKTRTGSLKRALHNAECQKTVTISKPCGKLTKPKPQAESKKKKKEGKKQEKMLD",
# VATB2_HUMAN
"MALRAMRGIVNGAAPELPVPTGGPAVGAREQALAVSRNYLSQPRLTYKTVSGVNGPLVILDHVKFPRYAEIVHLTLPDGTKRSGQVLEVSGSKAVVQVFEGTSGIDAKKTSCEFTGDILRTPVSEDMLGRVFNGSGKPIDRGPVVLAEDFLDIMGQPINPQCRIYPEEMIQTGISAIDGMNSIARGQKIPIFSAAGLPHNEIAAQICRQAGLVKKSKDVVDYSEENFAIVFAAMGVNMETARFFKSDFEENGSMDNVCLFLNLANDPTIERIITPRLALTTAEFLAYQCEKHVLVILTDMSSYAEALREVSAAREEVPGRRGFPGYMYTDLATIYERAGRVEGRNGSITQIPILTMPNDDITHPIPDLTGYITEGQIYVDRQLHNRQIYPPINVLPSLSRLMKSAIGEGMTRKDHADVSNQLYACYAIGKDVQAMKAVVGEEALTSDDLLYLEFLQKFERNFIAQGPYENRTVFETLDIGWQLLRIFPKEMLKRIPQSTLSEFYPRDSAKH",
# VATC1_HUMAN
"MTEFWLISAPGEKTCQQTWEKLHAATSKNNNLAVTSKFNIPDLKVGTLDVLVGLSDELAKLDAFVEGVVKKVAQYMADVLEDSKDKVQENLLANGVDLVTYITRFQWDMAKYPIKQSLKNISEIIAKGVTQIDNDLKSRASAYNNLKGNLQNLERKNAGSLLTRSLAEIVKKDDFVLDSEYLVTLLVVVPKLNHNDWIKQYETLAEMVVPRSSNVLSEDQDSYLCNVTLFRKAVDDFRHKARENKFIVRDFQYNEEEMKADKEEMNRLSTDKKKQFGPLVRWLKVNFSEAFIAWIHVKALRVFVESVLRYGLPVNFQAMLLQPNKKTLKKLREVLHELYKHLDSSAAAIIDAPMDIPGLNLSQQEYYPYVYYKIDCNLLEFK",
# FLNA_HUMAN
"MSSSHSRAGQSAAGAAPGGGVDTRDAEMPATEKDLAEDAPWKKIQQNTFTRWCNEHLKCVSKRIANLQTDLSDGLRLIALLEVLSQKKMHRKHNQRPTFRQMQLENVSVALEFLDRESIKLVSIDSKAIVDGNLKLILGLIWTLILHYSISMPMWDEEEDEEAKKQTPKQRLLGWIQNKLPQLPITNFSRDWQSGRALGALVDSCAPGLCPDWDSWDASKPVTNAREAMQQADDWLGIPQVITPEEIVDPNVDEHSVMTYLSQFPKAKLKPGAPLRPKLNPKKARAYGPGIEPTGNMVKKRAEFTVETRSAGQGEVLVYVEDPAGHQEEAKVTANNDKNRTFSVWYVPEVTGTHKVTVLFAGQHIAKSPFEVYVDKSQGDASKVTAQGPGLEPSGNIANKTTYFEIFTAGAGTGEVEVVIQDPMGQKGTVEPQLEARGDSTYRCSYQPTMEGVHTVHVTFAGVPIPRSPYTVTVGQACNPSACRAVGRGLQPKGVRVKETADFKVYTKGAGSGELKVTVKGPKGEERVKQKDLGDGVYGFEYYPMVPGTYIVTITWGGQNIGRSPFEVKVGTECGNQKVRAWGPGLEGGVVGKSADFVVEAIGDDVGTLGFSVEGPSQAKIECDDKGDGSCDVRYWPQEAGEYAVHVLCNSEDIRLSPFMADIRDAPQDFHPDRVKARGPGLEKTGVAVNKPAEFTVDAKHGGKAPLRVQVQDNEGCPVEALVKDNGNGTYSCSYVPRKPVKHTAMVSWGGVSIPNSPFRVNVGAGSHPNKVKVYGPGVAKTGLKAHEPTYFTVDCAEAGQGDVSIGIKCAPGVVGPAEADIDFDIIRNDNDTFTVKYTPRGAGSYTIMVLFADQATPTSPIRVKVEPSHDASKVKAEGPGLSRTGVELGKPTHFTVNAKAAGKGKLDVQFSGLTKGDAVRDVDIIDHHDNTYTVKYTPVQQGPVGVNVTYGGDPIPKSPFSVAVSPSLDLSKIKVSGLGEKVDVGKDQEFTVKSKGAGGQGKVASKIVGPSGAAVPCKVEPGLGADNSVVRFLPREEGPYEVEVTYDGVPVPGSPFPLEAVAPTKPSKVKAFGPGLQGGSAGSPARFTIDTKGAGTGGLGLTVEGPCEAQLECLDNGDGTCSVSYVPTEPGDYNINILFADTHIPGSPFKAHVVPCFDASKVKCSGPGLERATAGEVGQFQVDCSSAGSAELTIEICSEAGLPAEVYIQDHGDGTHTITYIPLCPGAYTVTIKYGGQPVPNFPSKLQVEPAVDTSGVQCYGPGIEGQGVFREATTEFSVDARALTQTGGPHVKARVANPSGNLTETYVQDRGDGMYKVEYTPYEEGLHSVDVTYDGSPVPSSPFQVPVTEGCDPSRVRVHGPGIQSGTTNKPNKFTVETRGAGTGGLGLAVEGPSEAKMSCMDNKDGSCSVEYIPYEAGTYSLNVTYGGHQVPGSPFKVPVHDVTDASKVKCSGPGLSPGMVRANLPQSFQVDTSKAGVAPLQVKVQGPKGLVEPVDVVDNADGTQTVNYVPSREGPYSISVLYGDEEVPRSPFKVKVLPTHDASKVKASGPGLNTTGVPASLPVEFTIDAKDAGEGLLAVQITDPEGKPKKTHIQDNHDGTYTVAYVPDVTGRYTILIKYGGDEIPFSPYRVRAVPTGDASKCTVTVSIGGHGLGAGIGPTIQIGEETVITVDTKAAGKGKVTCTVCTPDGSEVDVDVVENEDGTFDIFYTAPQPGKYVICVRFGGEHVPNSPFQVTALAGDQPSVQPPLRSQQLAPQYTYAQGGQQTWAPERPLVGVNGLDVTSLRPFDLVIPFTIKKGEITGEVRMPSGKVAQPTITDNKDGTVTVRYAPSEAGLHEMDIRYDNMHIPGSPLQFYVDYVNCGHVTAYGPGLTHGVVNKPATFTVNTKDAGEGGLSLAIEGPSKAEISCTDNQDGTCSVSYLPVLPGDYSILVKYNEQHVPGSPFTARVTGDDSMRMSHLKVGSAADIPINISETDLSLLTATVVPPSGREEPCLLKRLRNGHVGISFVPKETGEHLVHVKKNGQHVASSPIPVVISQSEIGDASRVRVSGQGLHEGHTFEPAEFIIDTRDAGYGGLSLSIEGPSKVDINTEDLEDGTCRVTYCPTEPGNYIINIKFADQHVPGSPFSVKVTGEGRVKESITRRRRAPSVANVGSHCDLSLKIPEISIQDMTAQVTSPSGKTHEAEIVEGENHTYCIRFVPAEMGTHTVSVKYKGQHVPGSPFQFTVGPLGEGGAHKVRAGGPGLERAEAGVPAEFSIWTREAGAGGLAIAVEGPSKAEISFEDRKDGSCGVAYVVQEPGDYEVSVKFNEEHIPDSPFVVPVASPSGDARRLTVSSLQESGLKVNQPASFAVSLNGAKGAIDAKVHSPSGALEECYVTEIDQDKYAVRFIPRENGVYLIDVKFNGTHIPGSPFKIRVGEPGHGGDPGLVSAYGAGLEGGVTGNPAEFVVNTSNAGAGALSVTIDGPSKVKMDCQECPEGYRVTYTPMAPGSYLISIKYGGPYHIGGSPFKAKVTGPRLVSNHSLHETSSVFVDSLTKATCAPQHGAPGPGPADASKVVAKGLGLSKAYVGQKSSFTVDCSKAGNNMLLVGVHGPRTPCEEILVKHVGSRLYSVSYLLKDKGEYTLVVKWGDEHIPGSPYRVVVP",
# NF1_HUMAN
"MAAHRPVEWVQAVVSRFDEQLPIKTGQQNTHTKVSTEHNKECLINISKYKFSLVISGLTTILKNVNNMRIFGEAAEKNLYLSQLIILDTLEKCLAGQPKDTMRLDETMLVKQLLPEICHFLHTCREGNQHAAELRNSASGVLFSLSCNNFNAVFSRISTRLQELTVCSEDNVDVHDIELLQYINVDCAKLKRLLKETAFKFKALKKVAQLAVINSLEKAFWNWVENYPDEFTKLYQIPQTDMAECAEKLFDLVDGFAESTKRKAAVWPLQIILLILCPEIIQDISKDVVDENNMNKKLFLDSLRKALAGHGGSRQLTESAAIACVKLCKASTYINWEDNSVIFLLVQSMVVDLKNLLFNPSKPFSRGSQPADVDLMIDCLVSCFRISPHNNQHFKICLAQNSPSTFHYVLVNSLHRIITNSALDWWPKIDAVYCHSVELRNMFGETLHKAVQGCGAHPAIRMAPSLTFKEKVTSLKFKEKPTDLETRSYKYLLLSMVKLIHADPKLLLCNPRKQGPETQGSTAELITGLVQLVPQSHMPEIAQEAMEALLVLHQLDSIDLWNPDAPVETFWEISSQMLFYICKKLTSHQMLSSTEILKWLREILICRNKFLLKNKQADRSSCHFLLFYGVGCDIPSSGNTSQMSMDHEELLRTPGASLRKGKGNSSMDSAAGCSGTPPICRQAQTKLEVALYMFLWNPDTEAVLVAMSCFRHLCEEADIRCGVDEVSVHNLLPNYNTFMEFASVSNMMSTGRAALQKRVMALLRRIEHPTAGNTEAWEDTHAKWEQATKLILNYPKAKMEDGQAAESLHKTIVKRRMSHVSGGGSIDLSDTDSLQEWINMTGFLCALGGVCLQQRSNSGLATYSPPMGPVSERKGSMISVMSSEGNADTPVSKFMDRLLSLMVCNHEKVGLQIRTNVKDLVGLELSPALYPMLFNKLKNTISKFFDSQGQVLLTDTNTQFVEQTIAIMKNLLDNHTEGSSEHLGQASIETMMLNLVRYVRVLGNMVHAIQIKTKLCQLVEVMMARRDDLSFCQEMKFRNKMVEYLTDWVMGTSNQAADDDVKCLTRDLDQASMEAVVSLLAGLPLQPEEGDGVELMEAKSQLFLKYFTLFMNLLNDCSEVEDESAQTGGRKRGMSRRLASLRHCTVLAMSNLLNANVDSGLMHSIGLGYHKDLQTRATFMEVLTKILQQGTEFDTLAETVLADRFERLVELVTMMGDQGELPIAMALANVVPCSQWDELARVLVTLFDSRHLLYQLLWNMFSKEVELADSMQTLFRGNSLASKIMTFCFKVYGATYLQKLLDPLLRIVITSSDWQHVSFEVDPTRLEPSESLEENQRNLLQMTEKFFHAIISSSSEFPPQLRSVCHCLYQATCHSLLNKATVKEKKENKKSVVSQRFPQNSIGAVGSAMFLRFINPAIVSPYEAGILDKKPPPRIERGLKLMSKILQSIANHVLFTKEEHMRPFNDFVKSNFDAARRFFLDIASDCPTSDAVNHSLSFISDGNVLALHRLLWNNQEKIGQYLSSNRDHKAVGRRPFDKMATLLAYLGPPEHKPVADTHWSSLNLTSSKFEEFMTRHQVHEKEEFKALKTLSIFYQAGTSKAGNPIFYYVARRFKTGQINGDLLIYHVLLTLKPYYAKPYEIVVDLTHTGPSNRFKTDFLSKWFVVFPGFAYDNVSAVYIYNCNSWVREYTKYHERLLTGLKGSKRLVFIDCPGKLAEHIEHEQQKLPAATLALEEDLKVFHNALKLAHKDTKVSIKVGSTAVQVTSAERTKVLGQSVFLNDIYYASEIEEICLVDENQFTLTIANQGTPLTFMHQECEAIVQSIIHIRTRWELSQPDSIPQHTKIRPKDVPGTLLNIALLNLGSSDPSLRSAAYNLLCALTCTFNLKIEGQLLETSGLCIPANNTLFIVSISKTLAANEPHLTLEFLEECISGFSKSSIELKHLCLEYMTPWLSNLVRFCKHNDDAKRQRVTAILDKLITMTINEKQMYPSIQAKIWGSLGQITDLLDVVLDSFIKTSATGGLGSIKAEVMADTAVALASGNVKLVSSKVIGRMCKIIDKTCLSPTPTLEQHLMWDDIAILARYMLMLSFNNSLDVAAHLPYLFHVVTFLVATGPLSLRASTHGLVINIIHSLCTCSQLHFSEETKQVLRLSLTEFSLPKFYLLFGISKVKSAAVIAFRSSYRDRSFSPGSYERETFALTSLETVTEALLEIMEACMRDIPTCKWLDQWTELAQRFAFQYNPSLQPRALVVFGCISKRVSHGQIKQIIRILSKALESCLKGPDTYNSQVLIEATVIALTKLQPLLNKDSPLHKALFWVAVAVLQLDEVNLYSAGTALLEQNLHTLDSLRIFNDKSPEEVFMAIRNPLEWHCKQMDHFVGLNFNSNFNFALVGHLLKGYRHPSPAIVARTVRILHTLLTLVNKHRNCDKFEVNTQSVAYLAALLTVSEEVRSRCSLKHRKSLLLTDISMENVPMDTYPIHHGDPSYRTLKETQPWSSPKGSEGYLAATYPTVGQTSPRARKSMSLDMGQPSQANTKKLLGTRKSFDHLISDTKAPKRQEMESGITTPPKMRRVAETDYEMETQRISSSQQHPHLRKVSVSESNVLLDEEVLTDPKIQALLLTVLATLVKYTTDEFDQRILYEYLAEASVVFPKVFPVVHNLLDSKINTLLSLCQDPNLLNPIHGIVQSVVYHEESPPQYQTSYLQSFGFNGLWRFAGPFSKQTQIPDYAELIVKFLDALIDTYLPGIDEETSEESLLTPTSPYPPALQSQLSITANLNLSNSMTSLATSQHSPGIDKENVELSPTTGHCNSGRTRHGSASQVQKQRSAGSFKRNSIKKIV",
# S1PR1_HUMAN
"MGPTSVPLVKAHRSSVSDYVNYDIIVRHYNYTGKLNISADKENSIKLTSVVFILICCFIILENIFVLLTIWKTKKFHRPMYYFIGNLALSDLLAGVAYTANLLLSGATTYKLTPAQWFLREGSMFVALSASVFSLLAIAIERYITMLKMKLHNGSNNFRLFLLISACWVISLILGGLPIMGWNCISALSSCSTVLPLYHKHYILFCTTVFTLLLLSIVILYCRIYSLVRTRSRRLTFRKNISKASRSSEKSLALLKTVIIVLSVFIACWAPLFILLLLDVGCKVKTCDILFRAEYFLVLAVLNSGTNPIIYTLTNKEMRRAFIRIMSCCKCPSGDSAGKFKRPIIAGMEFSRSKSDNSSHPQKDEGDNPETIMSSGNVNSSS",
# CNR1_HUMAN
"MKSILDGLADTTFRTITTDLLYVGSNDIQYEDIKGDMASKLGYFPQKFPLTSFRGSPFQEKMTAGDNPQLVPADQVNITEFYNKSLSSFKENEENIQCGENFMDIECFMVLNPSQQLAIAVLSLTLGTFTVLENLLVLCVILHSRSLRCRPSYHFIGSLAVADLLGSVIFVYSFIDFHVFHRKDSRNVFLFKLGGVTASFTASVGSLFLTAIDRYISIHRPLAYKRIVTRPKAVVAFCLMWTIAIVIAVLPLLGWNCEKLQSVCSDIFPHIDETYLMFWIGVTSVLLLFIVYAYMYILWKAHSHAVRMIQRGTQKSIIIHTSEDGKVQVTRPDQARMDIRLAKTLVLILVVLIICWGPLLAIMVYDVFGKMNKLIKTVFAFCSMLCLLNSTVNPIIYALRSKDLRHAFRSMFPSCEGTAQPLDNSMGDSDCLHKHANNAASVHRAAESCIKSTVKIAKVTMSVSTDTSAEAL",
# SYT1_HUMAN
"MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEKFMNELHKIPLPPWALIAIAIVAVLLVLTCCFCICKKCLFKKKNKKKGKEKGGKNAINMKDVKDLGKTMKDQALKDDDAETGLTDGEEKEEPKEEEKLGKLQYSLDYDFQNNQLLVGIIQAAELPALDMGGTSDPYVKVFLLPDKKKKFETKVHRKTLNPVFNEQFTFKVPYSELGGKTLVMAVYDFDRFSKHDIIGEFKVPMNTVDFGHVTEEWRDLQSAEKEEQEKLGDICFSLRYVPTAGKLTVVILEAKNLKKMDVGGLSDPYVKIHLMQNGKRLKKKKTTIKKNTLNPYYNESFSFEVPFEQIQKVQVVVTVLDYDKIGKNDAIGKVFVGYNSTGAELRHWSDMLANPRRPIAQWHTLQVEEEVDAMLAVKK",
# DRD1_HUMAN
"MRTLNTSAMDGTGLVVERDFSVRILTACFLSLLILSTLLGNTLVCAAVIRFRHLRSKVTNFFVISLAVSDLLVAVLVMPWKAVAEIAGFWPFGSFCNIWVAFDIMCSTASILNLCVISVDRYWAISSPFRYERKMTPKAAFILISVAWTLSVLISFIPVQLSWHKAKPTSPSDGNATSLAETIDNCDSSLSRTYAISSSVISFYIPVAIMIVTYTRIYRIAQKQIRRIAALERAAVHAKNCQTTTGNGKPVECSQPESSFKMSFKRETKVLKTLSVIMGVFVCCWLPFFILNCILPFCGSGETQPFCIDSNTFDVFVWFGWANSSLNPIIYAFNADFRKAFSTLLGCYRLCPATNNAIETVSINNNGAAMFSSHHEPRGSISKECNLVYLIPHAVGSSEDLKKEEAAGIARPLEKLSPALSVILDYDTDVSLEKIQPITQNGQHPT",
# FGF7_HUMAN
"MHKWILTWILPTLLYRSCFHIICLVGTISLACNDMTPEQMATNVNCSSPERHTRSYDYMEGGDIRVRRLFCRTQWYLRIDKRGKVKGTQEMKNNYNIMEIRTVAVGIVAIKGVESEFYLAMNKEGKLYAKKECNEDCNFKELILENHYNTYASAKWTHNGGEMFVALNQKGIPVRGKKTKKEQKTAHFLPMAIT",
# VDAC1_HUMAN
"MAVPPTYADLGKSARDVFTKGYGFGLIKLDLKTKSENGLEFTSSGSANTETTKVTGSLETKYRWTEYGLTFTEKWNTDNTLGTEITVEDQLARGLKLTFDSSFSPNTGKKNAKIKTGYKREHINLGCDMDFDIAGPSIRGALVLGYEGWLAGYQMNFETAKSRVTQSNFAVGYKTDEFQLHTNVNDGTEFGGSIYQKVNKKLETAVNLAWTAGNSNTRFGIAAKYQIDPDACFSAKVNNSSLIGLGYTQTLKPGIKLTLSALLDGKNVNAGGHKLGLGLEFQA",
# DRD4_HUMAN
"MGNRSTADADGLLAGRGPAAGASAGASAGLAGQGAAALVGGVLLIGAVLAGNSLVCVSVATERALQTPTNSFIVSLAAADLLLALLVLPLFVYSEVQGGAWLLSPRLCDALMAMDVMLCTASIFNLCAISVDRFVAVAVPLRYNRQGGSRRQLLLIGATWLLSAAVAAPVLCGLNDVRGRDPAVCRLEDRDYVVYSSVCSFFLPCPLMLLLYWATFRGLQRWEVARRAKLHGRAPRRPSGPGPPSPTPPAPRLPQDPCGPDCAPPAPGLPRGPCGPDCAPAAPSLPQDPCGPDCAPPAPGLPPDPCGSNCAPPDAVRAAALPPQTPPQTRRRRRAKITGRERKAMRVLPVVVGAFLLCWTPFFVVHITQALCPACSVPPRLVSAVTWLGYVNSALNPVIYTVFNAEFRNVFRKALRACC",
# DRD5_HUMAN
"MLPPGSNGTAYPGQFALYQQLAQGNAVGGSAGAPPLGPSQVVTACLLTLLIIWTLLGNVLVCAAIVRSRHLRANMTNVFIVSLAVSDLFVALLVMPWKAVAEVAGYWPFGAFCDVWVAFDIMCSTASILNLCVISVDRYWAISRPFRYKRKMTQRMALVMVGLAWTLSILISFIPVQLNWHRDQAASWGGLDLPNNLANWTPWEEDFWEPDVNAENCDSSLNRTYAISSSLISFYIPVAIMIVTYTRIYRIAQVQIRRISSLERAAEHAQSCRSSAACAPDTSLRASIKKETKVLKTLSVIMGVFVCCWLPFFILNCMVPFCSGHPEGPPAGFPCVSETTFDVFVWFGWANSSLNPVIYAFNADFQKVFAQLLGCSHFCSRTPVETVNISNELISYNQDIVFHKEIAAAYIHMMPNAVTPGNREVDNDEEEGPFDRMFQIYQTSPDGDPVAESVWELDCEGEISLDKITPFTPNGFH",
# COMT_HUMAN
"MPEAPPLLLAAVLLGLVLLVVLLLLLRHWGWGLCLIGWNEFILQPIHNLLMGDTKEQRILNHVLQHAEPGNAQSVLEAIDTYCEQKEWAMNVGDKKGKIVDAVIQEHQPSVLLELGAYCGYSAVRMARLLSPGARLITIEINPDCAAITQRMVDFAGVKDKVTLVVGASQDIIPQLKKKYDVDTLDMVFLDHWKDRYLPDTLLLEECGLLRKGTVLLADNVICPGAPDFLAHVRGSSCFECTHYQSFLEYREVVDGLEKAIYKGPGSEAGP",
# KCNA3_HUMAN
"MDERLSLLRSPPPPSARHRAHPPQRPASSGGAHTLVNHGYAEPAAGRELPPDMTVVPGDHLLEPEVADGGGAPPQGGCGGGGCDRYEPLPPSLPAAGEQDCCGERVVINISGLRFETQLKTLCQFPETLLGDPKRRMRYFDPLRNEYFFDRNRPSFDAILYYYQSGGRIRRPVNVPIDIFSEEIRFYQLGEEAMEKFREDEGFLREEERPLPRRDFQRQVWLLFEYPESSGPARGIAIVSVLVILISIVIFCLETLPEFRDEKDYPASTSQDSFEAAGNSTSGSRAGASSFSDPFFVVETLCIIWFSFELLVRFFACPSKATFSRNIMNLIDIVAIIPYFITLGTELAERQGNGQQAMSLAILRVIRLVRVFRIFKLSRHSKGLQILGQTLKASMRELGLLIFFLFIGVILFSSAVYFAEADDPTSGFSSIPDAFWWAVVTMTTVGYGDMHPVTIGGKIVGSLCAIAGVLTIALPVPVIVSNFNYFYHRETEGEEQSQYMHVGSCQHLSSSAEELRKARSNSTLSKSEYMVIEEGGMNHSAFPQTPFKTGNSTATCTTNNNPNSCVNIKKIFTDV",
# ACES_HUMAN
"MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGGRLRGIRLKTPGGPVSAFLGIPFAEPPMGPRRFLPPEPKQPWSGVVDATTFQSVCYQYVDTLYPGFEGTEMWNPNRELSEDCLYLNVWTPYPRPTSPTPVLVWIYGGGFYSGASSLDVYDGRFLVQAERTVLVSMNYRVGAFGFLALPGSREAPGNVGLLDQRLALQWVQENVAAFGGDPTSVTLFGESAGAASVGMHLLSPPSRGLFHRAVLQSGAPNGPWATVGMGEARRRATQLAHLVGCPPGGTGGNDTELVACLRTRPAQVLVNHEWHVLPQESVFRFSFVPVVDGDFLSDTPEALINAGDFHGLQVLVGVVKDEGSYFLVYGAPGFSKDNESLISRAEFLAGVRVGVPQVSDLAAEAVVLHYTDWLHPEDPARLREALSDVVGDHNVVCPVAQLAGRLAAQGARVYAYVFEHRASTLSWPLWMGVPHGYEIEFIFGIPLDPSRNYTAEEKIFAQRLMRYWANFARTGDPNEPRDPKAPQWPPYTAGAQQYVSLDLRPLEVRRGLRAQACAFWNRFLPKLLSATDTLDEAERQWKAEFHRWSSYMVHWKNQFDHYSKQDRCSDL",
# KCNA4_HUMAN
"MEVAMVSAESSGCNSHMPYGYAAQARARERERLAHSRAAAAAAVAAATAAVEGSGGSGGGSHHHHQSRGACTSHDPQSSRGSRRRRRQRSEKKKAHYRQSSFPHCSDLMPSGSEEKILRELSEEEEDEEEEEEEEEEGRFYYSEDDHGDECSYTDLLPQDEGGGGYSSVRYSDCCERVVINVSGLRFETQMKTLAQFPETLLGDPEKRTQYFDPLRNEYFFDRNRPSFDAILYYYQSGGRLKRPVNVPFDIFTEEVKFYQLGEEALLKFREDEGFVREEEDRALPENEFKKQIWLLFEYPESSSPARGIAIVSVLVILISIVIFCLETLPEFRDDRDLVMALSAGGHGGLLNDTSAPHLENSGHTIFNDPFFIVETVCIVWFSFEFVVRCFACPSQALFFKNIMNIIDIVSILPYFITLGTDLAQQQGGGNGQQQQAMSFAILRIIRLVRVFRIFKLSRHSKGLQILGHTLRASMRELGLLIFFLFIGVILFSSAVYFAEADEPTTHFQSIPDAFWWAVVTMTTVGYGDMKPITVGGKIVGSLCAIAGVLTIALPVPVIVSNFNYFYHRETENEEQTQLTQNAVSCPYLPSNLLKKFRSSTSSSLGDKSEYLEMEEGVKESLCAKEEKCQGKGDDSETDKNNCSNAKAVETDV",
# ROA2_HUMAN
"MEKTLETVPLERKKREKEQFRKLFIGGLSFETTEESLRNYYEQWGKLTDCVVMRDPASKRSRGFGFVTFSSMAEVDAAMAARPHSIDGRVVEPKRAVAREESGKPGAHVTVKKLFVGGIKEDTEEHHLRDYFEEYGKIDTIEIITDRQSGKKRGFGFVTFDDHDPVDKIVLQKYHTINGHNAEVRKALSRQEMQEVQSSRSGRGGNFGFGDSRGGGGNFGPGPGSNFRGGSDGYGSGRGFGDGYNGYGGGPGGGNFGGSPGYGGGRGGYGGGGPGYGNQGGGYGGGYDNYGGGNYGSGNYNDFGNYNQQPSNYGPMKSGNFGGSRNMGGPYGGGNYGPGGSGGSGGYGGRSRY",
# CALB2_HUMAN
"MAGPQQQPPYLHLAELTASQFLEIWKHFDADGNGYIEGKELENFFQELEKARKGSGMMSKSDNFGEKMKEFMQKYDKNSDGKIEMAELAQILPTEENFLLCFRQHVGSSAEFMEAWRKYDTDRSGYIEANELKGFLSDLLKKANRPYDEPKLQEYTQTILRMFDLNGDGKLGLSEMSRLLPVQENFLLKFQGMKLTSEEFNAIFTFYDKDRSGYIDEHELDALLKDLYEKNKKEMNIQQLTNYRKSVMSLAEAGKLYRKDLEIVLCSEPPM",
# SFPQ_HUMAN
"MSRDRFRSRGGGGGGFHRRGGGGGRGGLHDFRSPPPGMGLNQNRGPMGPGPGQSGPKPPIPPPPPHQQQQQPPPQQPPPQQPPPHQPPPHPQPHQQQQPPPPPQDSSKPVVAQGPGPAPGVGSAPPASSSAPPATPPTSGAPPGSGPGPTPTPPPAVTSAPPGAPPPTPPSSGVPTTPPQAGGPPPPPAAVPGPGPGPKQGPGPGGPKGGKMPGGPKPGGGPGLSTPGGHPKPPHRGGGEPRGGRQHHPPYHQQHHQGPPPGGPGGRSEEKISDSEGFKANLSLLRRPGEKTYTQRCRLFVGNLPADITEDEFKRLFAKYGEPGEVFINKGKGFGFIKLESRALAEIAKAELDDTPMRGRQLRVRFATHAAALSVRNLSPYVSNELLEEAFSQFGPIERAVVIVDDRGRSTGKGIVEFASKPAARKAFERCSEGVFLLTTTPRPVIVEPLEQLDDEDGLPEKLAQKNPMYQKERETPPRFAQHGTFEYEYSQRWKSLDEMEKQQREQVEKNMKDAKDKLESEMEDAYHEHQANLLRQDLMRRQEELRRMEELHNQEMQKRKEMQLRQEEERRRREEEMMIRQREMEEQMRRQREESYSRMGYMDPRERDMRMGGGGAMNMGDPYGSGGQKFPPLGGGGGIGYEANPGVPPATMSGSMMGSDMRTERFGQGGAGPVGGQGPRGMGPGTPAGYGRGREEYEGPNKKPRF",
# RS3_HUMAN
"MAVQISKKRKFVADGIFKAELNEFLTRELAEDGYSGVEVRVTPTRTEIIILATRTQNVLGEKGRRIRELTAVVQKRFGFPEGSVELYAEKVATRGLCAIAQAESLRYKLLGGLAVRRACYGVLRFIMESGAKGCEVVVSGKLRGQRAKSMKFVDGLMIHSGDPVNYYVDTAVRHVLLRQGVLGIKVKIMLPWDPTGKIGPKKPLPDHVSIVEPKDEILPTTPISEQKGGKPEPPAMPQPVPTA",
# GLRA1_HUMAN
"MYSFNTLRLYLWETIVFFSLAASKEAEAARSAPKPMSPSDFLDKLMGRTSGYDARIRPNFKGPPVNVSCNIFINSFGSIAETTMDYRVNIFLRQQWNDPRLAYNEYPDDSLDLDPSMLDSIWKPDLFFANEKGAHFHEITTDNKLLRISRNGNVLYSIRITLTLACPMDLKNFPMDVQTCIMQLESFGYTMNDLIFEWQEQGAVQVADGLTLPQFILKEEKDLRYCTKHYNTGKFTCIEARFHLERQMGYYLIQMYIPSLLIVILSWISFWINMDAAPARVGLGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFSALLEYAAVNFVSRQHKELLRFRRKRRHHKSPMLNLFQEDEAGEGRFNFSAYGMGPACLQAKDGISVKGANNSNTTNPPPAPSKSPEEMRKLFIQRAKKIDKISRIGFPMAFLIFNMFYWIIYKIVRREDVHNQ",
# GLRA2_HUMAN
"MNRQLVNILTALFAFFLETNHFRTAFCKDHDSRSGKQPSQTLSPSDFLDKLMGRTSGYDARIRPNFKGPPVNVTCNIFINSFGSVTETTMDYRVNIFLRQQWNDSRLAYSEYPDDSLDLDPSMLDSIWKPDLFFANEKGANFHDVTTDNKLLRISKNGKVLYSIRLTLTLSCPMDLKNFPMDVQTCTMQLESFGYTMNDLIFEWLSDGPVQVAEGLTLPQFILKEEKELGYCTKHYNTGKFTCIEVKFHLERQMGYYLIQMYIPSLLIVILSWVSFWINMDAAPARVALGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFAALLEYAAVNFVSRQHKEFLRLRRRQKRQNKEEDVTRESRFNFSGYGMGHCLQVKDGTAVKATPANPLPQPPKDGDAIKKKFVDRAKRIDTISRAAFPLAFLIFNIFYWITYKIIRHEDVHKK",
# CBLN1_HUMAN
"MLGVLELLLLGAAWLAGPARGQNETEPIVLEGKCLVVCDSNPTSDPTGTALGISVRSGSAKVAFSAIRSTNHEPSEMSNRTMIIYFDQVLVNIGNNFDSERSTFIAPRKGIYSFNFHVVKVYNRQTIQVSLMLNGWPVISAFAGDQDVTREAASNGVLIQMEKGDRAYLKLERGNLMGGWKYSTFSGFLVFPL",
# KS6B1_HUMAN
"MRRRRRRDGFYPAPDFRDREAEDMAGVFDIDLDQPEDAGSEDELEEGGQLNESMDHGGVGPYELGMEHCEKFEISETSVNRGPEKIRPECFELLRVLGKGGYGKVFQVRKVTGANTGKIFAMKVLKKAMIVRNAKDTAHTKAERNILEEVKHPFIVDLIYAFQTGGKLYLILEYLSGGELFMQLEREGIFMEDTACFYLAEISMALGHLHQKGIIYRDLKPENIMLNHQGHVKLTDFGLCKESIHDGTVTHTFCGTIEYMAPEILMRSGHNRAVDWWSLGALMYDMLTGAPPFTGENRKKTIDKILKCKLNLPPYLTQEARDLLKKLLKRNAASRLGAGPGDAGEVQAHPFFRHINWEELLARKVEPPFKPLLQSEEDVSQFDSKFTRQTPVDSPDDSTLSESANQVFLGFTYVAPSVLESVKEKFSFEPKIRSPRRFIGSPRTPVSPVKFSPGDFWGRGASASTANPQTPVEYPMETSGIEQMDVTMSGEASAPLPIRQPNSGPYKKQAFPMISKRPEHLRMNL",
# PTPRD_HUMAN
"MVHVARLLLLLLTFFLRTDAETPPRFTRTPVDQTGVSGGVASFICQATGDPRPKIVWNKKGKKVSNQRFEVIEFDDGSGSVLRIQPLRTPRDEAIYECVASNNVGEISVSTRLTVLREDQIPRGFPTIDMGPQLKVVERTRTATMLCAASGNPDPEITWFKDFLPVDTSNNNGRIKQLRSESIGGTPIRGALQIEQSEESDQGKYECVATNSAGTRYSAPANLYVRELREVRRVPPRFSIPPTNHEIMPGGSVNITCVAVGSPMPYVKWMLGAEDLTPEDDMPIGRNVLELNDVRQSANYTCVAMSTLGVIEAIAQITVKALPKPPGTPVVTESTATSITLTWDSGNPEPVSYYIIQHKPKNSEELYKEIDGVATTRYSVAGLSPYSDYEFRVVAVNNIGRGPPSEPVLTQTSEQAPSSAPRDVQARMLSSTTILVQWKEPEEPNGQIQGYRVYYTMDPTQHVNNWMKHNVADSQITTIGNLVPQKTYSVKVLAFTSIGDGPLSSDIQVITQTGVPGQPLNFKAEPESETSILLSWTPPRSDTIANYELVYKDGEHGEEQRITIEPGTSYRLQGLKPNSLYYFRLAARSPQGLGASTAEISARTMQSKPSAPPQDISCTSPSSTSILVSWQPPPVEKQNGIITEYSIKYTAVDGEDDKPHEILGIPSDTTKYLLEQLEKWTEYRITVTAHTDVGPGPESLSVLIRTNEDVPSGPPRKVEVEAVNSTSVKVSWRSPVPNKQHGQIRGYQVHYVRMENGEPKGQPMLKDVMLADAQWEFDDTTEHDMIISGLQPETSYSLTVTAYTTKGDGARSKPKLVSTTGAVPGKPRLVINHTQMNTALIQWHPPVDTFGPLQGYRLKFGRKDMEPLTTLEFSEKEDHFTATDIHKGASYVFRLSARNKVGFGEEMVKEISIPEEVPTGFPQNLHSEGTTSTSVQLSWQPPVLAERNGIITKYTLLYRDINIPLLPMEQLIVPADTTMTLTGLKPDTTYDVKVRAHTSKGPGPYSPSVQFRTLPVDQVFAKNFHVKAVMKTSVLLSWEIPENYNSAMPFKILYDDGKMVEEVDGRATQKLIVNLKPEKSYSFVLTNRGNSAGGLQHRVTAKTAPDVLRTKPAFIGKTNLDGMITVQLPEVPANENIKGYYIIIVPLKKSRGKFIKPWESPDEMELDELLKEISRKRRSIRYGREVELKPYIAAHFDVLPTEFTLGDDKHYGGFTNKQLQSGQEYVFFVLAVMEHAESKMYATSPYSDPVVSMDLDPQPITDEEEGLIWVVGPVLAVVFIICIVIAILLYKRKRAESDSRKSSIPNNKEIPSHHPTDPVELRRLNFQTPGMASHPPIPILELADHIERLKANDNLKFSQEYESIDPGQQFTWEHSNLEVNKPKNRYANVIAYDHSRVLLSAIEGIPGSDYVNANYIDGYRKQNAYIATQGSLPETFGDFWRMIWEQRSATVVMMTKLEERSRVKCDQYWPSRGTETHGLVQVTLLDTVELATYCVRTFALYKNGSSEKREVRQFQFTAWPDHGVPEHPTPFLAFLRRVKTCNPPDAGPMVVHCSAGVGRTGCFIVIDAMLERIKHEKTVDIYGHVTLMRAQRNYMVQTEDQYIFIHDALLEAVTCGNTEVPARNLYAYIQKLTQIETGENVTGMELEFKRLASSKAHTSRFISANLPCNKFKNRLVNIMPYESTRVCLQPIRGVEGSDYINASFIDGYRQQKAYIATQGPLAETTEDFWRMLWEHNSTIVVMLTKLREMGREKCHQYWPAERSARYQYFVVDPMAEYNMPQYILREFKVTDARDGQSRTVRQFQFTDWPEQGVPKSGEGFIDFIGQVHKTKEQFGQDGPISVHCSAGVGRTGVFITLSIVLERMRYEGVVDIFQTVKMLRTQRPAMVQTEDQYQFSYRAALEYLGSFDHYAT",
# PTPRZ_HUMAN
"MRILKRFLACIQLLCVCRLDWANGYYRQQRKLVEEIGWSYTGALNQKNWGKKYPTCNSPKQSPINIDEDLTQVNVNLKKLKFQGWDKTSLENTFIHNTGKTVEINLTNDYRVSGGVSEMVFKASKITFHWGKCNMSSDGSEHSLEGQKFPLEMQIYCFDADRFSSFEEAVKGKGKLRALSILFEVGTEENLDFKAIIDGVESVSRFGKQAALDPFILLNLLPNSTDKYYIYNGSLTSPPCTDTVDWIVFKDTVSISESQLAVFCEVLTMQQSGYVMLMDYLQNNFREQQYKFSRQVFSSYTGKEEIHEAVCSSEPENVQADPENYTSLLVTWERPRVVYDTMIEKFAVLYQQLDGEDQTKHEFLTDGYQDLGAILNNLLPNMSYVLQIVAICTNGLYGKYSDQLIVDMPTDNPELDLFPELIGTEEIIKEEEEGKDIEEGAIVNPGRDSATNQIRKKEPQISTTTHYNRIGTKYNEAKTNRSPTRGSEFSGKGDVPNTSLNSTSQPVTKLATEKDISLTSQTVTELPPHTVEGTSASLNDGSKTVLRSPHMNLSGTAESLNTVSITEYEEESLLTSFKLDTGAEDSSGSSPATSAIPFISENISQGYIFSSENPETITYDVLIPESARNASEDSTSSGSEESLKDPSMEGNVWFPSSTDITAQPDVGSGRESFLQTNYTEIRVDESEKTTKSFSAGPVMSQGPSVTDLEMPHYSTFAYFPTEVTPHAFTPSSRQQDLVSTVNVVYSQTTQPVYNGETPLQPSYSSEVFPLVTPLLLDNQILNTTPAASSSDSALHATPVFPSVDVSFESILSSYDGAPLLPFSSASFSSELFRHLHTVSQILPQVTSATESDKVPLHASLPVAGGDLLLEPSLAQYSDVLSTTHAASETLEFGSESGVLYKTLMFSQVEPPSSDAMMHARSSGPEPSYALSDNEGSQHIFTVSYSSAIPVHDSVGVTYQGSLFSGPSHIPIPKSSLITPTASLLQPTHALSGDGEWSGASSDSEFLLPDTDGLTALNISSPVSVAEFTYTTSVFGDDNKALSKSEIIYGNETELQIPSFNEMVYPSESTVMPNMYDNVNKLNASLQETSVSISSTKGMFPGSLAHTTTKVFDHEISQVPENNFSVQPTHTVSQASGDTSLKPVLSANSEPASSDPASSEMLSPSTQLLFYETSASFSTEVLLQPSFQASDVDTLLKTVLPAVPSDPILVETPKVDKISSTMLHLIVSNSASSENMLHSTSVPVFDVSPTSHMHSASLQGLTISYASEKYEPVLLKSESSHQVVPSLYSNDELFQTANLEINQAHPPKGRHVFATPVLSIDEPLNTLINKLIHSDEILTSTKSSVTGKVFAGIPTVASDTFVSTDHSVPIGNGHVAITAVSPHRDGSVTSTKLLFPSKATSELSHSAKSDAGLVGGGEDGDTDDDGDDDDDDRGSDGLSIHKCMSCSSYRESQEKVMNDSDTHENSLMDQNNPISYSLSENSEEDNRVTSVSSDSQTGMDRSPGKSPSANGLSQKHNDGKEENDIQTGSALLPLSPESKAWAVLTSDEESGSGQGTSDSLNENETSTDFSFADTNEKDADGILAAGDSEITPGFPQSPTSSVTSENSEVFHVSEAEASNSSHESRIGLAEGLESEKKAVIPLVIVSALTFICLVVLVGILIYWRKCFQTAHFYLEDSTSPRVISTPPTPIFPISDDVGAIPIKHFPKHVADLHASSGFTEEFETLKEFYQEVQSCTVDLGITADSSNHPDNKHKNRYINIVAYDHSRVKLAQLAEKDGKLTDYINANYVDGYNRPKAYIAAQGPLKSTAEDFWRMIWEHNVEVIVMITNLVEKGRRKCDQYWPADGSEEYGNFLVTQKSVQVLAYYTVRNFTLRNTKIKKGSQKGRPSGRVVTQYHYTQWPDMGVPEYSLPVLTFVRKAAYAKRHAVGPVVVHCSAGVGRTGTYIVLDSMLQQIQHEGTVNIFGFLKHIRSQRNYLVQTEEQYVFIHDTLVEAILSKETEVLDSHIHAYVNALLIPGPAGKTKLEKQFQLLSQSNIQQSDYSAALKQCNREKNRTSSIIPVERSRVGISSLSGEGTDYINASYIMGYYQSNEFIITQHPLLHTIKDFWRMIWDHNAQLVVMIPDGQNMAEDEFVYWPNKDEPINCESFKVTLMAEEHKCLSNEEKLIIQDFILEATQDDYVLEVRHFQCPKWPNPDSPISKTFELISVIKEEAANRDGPMIVHDEHGGVTAGTFCALTTLMHQLEKENSVDVYQVAKMINLMRPGVFADIEQYQFLYKVILSLVSTRQEENPSTSLDSNGAALPDGNIAESLESLV",
# COF1_HUMAN
"MASGVAVSDGVIKVFNDMKVRKSSTPEEVKKRKKAVLFCLSEDKKNIILEEGKEILVGDVGQTVDDPYATFVKMLPDKDCRYALYDATYETKESKKEDLVFIFWAPESAPLKSKMIYASSKDAIKKKLTGIKHELQANCYEEVKDRCTLAEKLGGSAVISLEGKPL",
# BDNF_HUMAN
"MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESVNGPKAGSRGLTSLADTFEHVIEELLDEDQKVRPNEENNKDADLYTSRVMLSSQVPLEPPLLFLLEEYKNYLDAANMSMRVRRHSDPARRGELSVCDSISEWVTAADKKTAVDMSGGTVTVLEKVPVSKGQLKQYFYETKCNPMGYTKEGCRGIDKRHWNSQCRTTQSYVRALTMDSKKRIGWRFIRIDTSCVCTLTIKRGR",
# IF4B_HUMAN
"MAASAKKKNKKGKTISLTDFLAEDGGTGGGSTYVSKPVSWADETDDLEGDVSTTWHSNDDDVYRAPPIDRSILPTAPRAAREPNIDRSRLPKSPPYTAFLGNLPYDVTEESIKEFFRGLNISAVRLPREPSNPERLKGFGYAEFEDLDSLLSALSLNEESLGNRRIRVDVADQAQDKDRDDRSFGRDRNRDSDKTDTDWRARPATDSFDDYPPRRGDDSFGDKYRDRYDSDRYRDGYRDGYRDGPRRDMDRYGGRDRYDDRGSRDYDRGYDSRIGSGRRAFGSGYRRDDDYRGGGDRYEDRYDRRDDRSWSSRDDYSRDDYRRDDRGPPQRPKLNLKPRSTPKEDDSSASTSQSTRAASIFGGAKPVDTAAREREVEERLQKEQEKLQRQLDEPKLERRPRERHPSWRSEETQERERSRTGSESSQTGTSTTSSRNARRRESEKSLENETLNKEEDCHSPTSKPPKPDQPLKVMPAPPPKENAWVKRSSNPPARSQSSDTEQQSPTSGGGKVAPAQPSEEGPGRKDENKVDGMNAPKGQTGNSSRGPGDGGNRDHWKESDRKDGKKDQDSRSAPEPKKPEENPASKFSSASKYAALSVDGEDENEGEDYAE",
# AT2B4_HUMAN
"MTNPSDRVLPANSMAESREGDFGCTVMELRKLMELRSRDALTQINVHYGGVQNLCSRLKTSPVEGLSGNPADLEKRRQVFGHNVIPPKKPKTFLELVWEALQDVTLIILEIAAIISLVLSFYRPAGEENELCGQVATTPEDENEAQAGWIEGAAILFSVIIVVLVTAFNDWSKEKQFRGLQCRIEQEQKFSIIRNGQLIQLPVAEIVVGDIAQVKYGDLLPADGILIQGNDLKIDESSLTGESDHVKKSLDKDPMLLSGTHVMEGSGRMVVTAVGVNSQTGIILTLLGVNEDDEGEKKKKGKKQGVPENRNKAKTQDGVALEIQPLNSQEGIDNEEKDKKAVKVPKKEKSVLQGKLTRLAVQIGKAGLLMSALTVFILILYFVIDNFVINRRPWLPECTPIYIQYFVKFFIIGITVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTMNRMTVVQAYIGGIHYRQIPSPDVFLPKVLDLIVNGISINSAYTSKILPPEKEGGLPRQVGNKTECALLGFVTDLKQDYQAVRNEVPEEKLYKVYTFNSVRKSMSTVIRNPNGGFRMYSKGASEIILRKCNRILDRKGEAVPFKNKDRDDMVRTVIEPMACDGLRTICIAYRDFDDTEPSWDNENEILTELTCIAVVGIEDPVRPEVPDAIAKCKQAGITVRMVTGDNINTARAIATKCGILTPGDDFLCLEGKEFNRLIRNEKGEVEQEKLDKIWPKLRVLARSSPTDKHTLVKGIIDSTVGEHRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFTSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTFASLALATEPPTESLLKRRPYGRNKPLISRTMMKNILGHAFYQLIVIFILVFAGEKFFDIDSGRKAPLHSPPSQHYTIVFNTFVLMQLFNEINSRKIHGEKNVFSGIYRNIIFCSVVLGTFICQIFIVEFGGKPFSCTSLSLSQWLWCLFIGIGELLWGQFISAIPTRSLKFLKEAGHGTTKEEITKDAEGLDEIDHAEMELRRGQILWFRGLNRIQTQIDVINTFQTGASFKGVLRRQNMGQHLDVKLVPSSSYIKVVKAFHSSLHESIQKPYNQKSIHSFMTHPEFAIEEELPRTPLLDEEEEENPDKASKFGTRVLLLDGEVTPYANTNNNAVDCNQVQLPQSDSSLQSLETSV",
# IP3KA_HUMAN
"MTLPGGPTGMARPGGARPCSPGLERAPRRSVGELRLLFEARCAAVAAAAAAGEPRARGAKRRGGQVPNGLPRAPPAPVIPQLTVTAEEPDVPPTSPGPPERERDCLPAAGSSHLQQPRRLSTSSVSSTGSSSLLEDSEDDLLSDSESRSRGNVQLEAGEDVGQKNHWQKIRTMVNLPVISPFKKRYAWVQLAGHTGSFKAAGTSGLILKRCSEPERYCLARLMADALRGCVPAFHGVVERDGESYLQLQDLLDGFDGPCVLDCKMGVRTYLEEELTKARERPKLRKDMYKKMLAVDPEAPTEEEHAQRAVTKPRYMQWREGISSSTTLGFRIEGIKKADGSCSTDFKTTRSREQVLRVFEEFVQGDEEVLRRYLNRLQQIRDTLEVSEFFRRHEVIGSSLLFVHDHCHRAGVWLIDFGKTTPLPDGQILDHRRPWEEGNREDGYLLGLDNLIGILASLAER",
# VAMP1_HUMAN
"MSAPAQPPAEGTEGTAPGGGPPGPPPNMTSNRRLQQTQAQVEEVVDIIRVNVDKVLERDQKLSELDDRADALQAGASQFESSAAKLKRKYWWKNCKMMIMLGAICAIIVVVIVIYFFT",
# SC6A2_HUMAN
"MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLLAPRDGDAQPRETWGKKIDFLLSVVGFAVDLANVWRFPYLCYKNGGGAFLIPYTLFLIIAGMPLFYMELALGQYNREGAATVWKICPFFKGVGYAVILIALYVGFYYNVIIAWSLYYLFSSFTLNLPWTDCGHTWNSPNCTDPKLLNGSVLGNHTKYSKYKFTPAAEFYERGVLHLHESSGIHDIGLPQWQLLLCLMVVVIVLYFSLWKGVKTSGKVVWITATLPYFVLFVLLVHGVTLPGASNGINAYLHIDFYRLKEATVWIDAATQIFFSLGAGFGVLIAFASYNKFDNNCYRDALLTSSINCITSFVSGFAIFSILGYMAHEHKVNIEDVATEGAGLVFILYPEAISTLSGSTFWAVVFFVMLLALGLDSSMGGMEAVITGLADDFQVLKRHRKLFTFGVTFSTFLLALFCITKGGIYVLTLLDTFAAGTSILFAVLMEAIGVSWFYGVDRFSNDIQQMMGFRPGLYWRLCWKFVSPAFLLFVVVVSIINFKPLTYDDYIFPPWANWVGWGIALSSMVLVPIYVIYKFLSTQGSLWERLAYGITPENEHHLVAQRDIRQFQLQHWLAI",
# LAMA2_HUMAN
"MPGAAGVLLLLLLSGGLGGVQAQRPQQQRQSQAHQQRGLFPAVLNLASNALITTNATCGEKGPEMYCKLVEHVPGQPVRNPQCRICNQNSSNPNQRHPITNAIDGKNTWWQSPSIKNGIEYHYVTITLDLQQVFQIAYVIVKAANSPRPGNWILERSLDDVEYKPWQYHAVTDTECLTLYNIYPRTGPPSYAKDDEVICTSFYSKIHPLENGEIHISLINGRPSADDPSPELLEFTSARYIRLRFQRIRTLNADLMMFAHKDPREIDPIVTRRYYYSVKDISVGGMCICYGHARACPLDPATNKSRCECEHNTCGDSCDQCCPGFHQKPWRAGTFLTKTECEACNCHGKAEECYYDENVARRNLSLNIRGKYIGGGVCINCTQNTAGINCETCTDGFFRPKGVSPNYPRPCQPCHCDPIGSLNEVCVKDEKHARRGLAPGSCHCKTGFGGVSCDRCARGYTGYPDCKACNCSGLGSKNEDPCFGPCICKENVEGGDCSRCKSGFFNLQEDNWKGCDECFCSGVSNRCQSSYWTYGKIQDMSGWYLTDLPGRIRVAPQQDDLDSPQQISISNAEARQALPHSYYWSAPAPYLGNKLPAVGGQLTFTISYDLEEEEEDTERVLQLMIILEGNDLSISTAQDEVYLHPSEEHTNVLLLKEESFTIHGTHFPVRRKEFMTVLANLKRVLLQITYSFGMDAIFRLSSVNLESAVSYPTDGSIAAAVEVCQCPPGYTGSSCESCWPRHRRVNGTIFGGICEPCQCFGHAESCDDVTGECLNCKDHTGGPYCDKCLPGFYGEPTKGTSEDCQPCACPLNIPSNNFSPTCHLDRSLGLICDGCPVGYTGPRCERCAEGYFGQPSVPGGSCQPCQCNDNLDFSIPGSCDSLSGSCLICKPGTTGRYCELCADGYFGDAVDAKNCQPCRCNAGGSFSEVCHSQTGQCECRANVQGQRCDKCKAGTFGLQSARGCVPCNCNSFGSKSFDCEESGQCWCQPGVTGKKCDRCAHGYFNFQEGGCTACECSHLGNNCDPKTGRCICPPNTIGEKCSKCAPNTWGHSITTGCKACNCSTVGSLDFQCNVNTGQCNCHPKFSGAKCTECSRGHWNYPRCNLCDCFLPGTDATTCDSETKKCSCSDQTGQCTCKVNVEGIHCDRCRPGKFGLDAKNPLGCSSCYCFGTTTQCSEAKGLIRTWVTLKAEQTILPLVDEALQHTTTKGIVFQHPEIVAHMDLMREDLHLEPFYWKLPEQFEGKKLMAYGGKLKYAIYFEAREETGFSTYNPQVIIRGGTPTHARIIVRHMAAPLIGQLTRHEIEMTEKEWKYYGDDPRVHRTVTREDFLDILYDIHYILIKATYGNFMRQSRISEISMEVAEQGRGTTMTPPADLIEKCDCPLGYSGLSCEACLPGFYRLRSQPGGRTPGPTLGTCVPCQCNGHSSLCDPETSICQNCQHHTAGDFCERCALGYYGIVKGLPNDCQQCACPLISSSNNFSPSCVAEGLDDYRCTACPRGYEGQYCERCAPGYTGSPGNPGGSCQECECDPYGSLPVPCDPVTGFCTCRPGATGRKCDGCKHWHAREGWECVFCGDECTGLLLGDLARLEQMVMSINLTGPLPAPYKMLYGLENMTQELKHLLSPQRAPERLIQLAEGNLNTLVTEMNELLTRATKVTADGEQTGQDAERTNTRAKSLGEFIKELARDAEAVNEKAIKLNETLGTRDEAFERNLEGLQKEIDQMIKELRRKNLETQKEIAEDELVAAEALLKKVKKLFGESRGENEEMEKDLREKLADYKNKVDDAWDLLREATDKIREANRLFAVNQKNMTALEKKKEAVESGKRQIENTLKEGNDILDEANRLADEINSIIDYVEDIQTKLPPMSEELNDKIDDLSQEIKDRKLAEKVSQAESHAAQLNDSSAVLDGILDEAKNISFNATAAFKAYSNIKDYIDEAEKVAKEAKDLAHEATKLATGPRGLLKEDAKGCLQKSFRILNEAKKLANDVKENEDHLNGLKTRIENADARNGDLLRTLNDTLGKLSAIPNDTAAKLQAVKDKARQANDTAKDVLAQITELHQNLDGLKKNYNKLADSVAKTNAVVKDPSKNKIIADADATVKNLEQEADRLIDKLKPIKELEDNLKKNISEIKELINQARKQANSIKVSVSSGGDCIRTYKPEIKKGSYNNIVVNVKTAVADNLLFYLGSAKFIDFLAIEMRKGKVSFLWDVGSGVGRVEYPDLTIDDSYWYRIVASRTGRNGTISVRALDGPKASIVPSTHHSTSPPGYTILDVDANAMLFVGGLTGKLKKADAVRVITFTGCMGETYFDNKPIGLWNFREKEGDCKGCTVSPQVEDSEGTIQFDGEGYALVSRPIRWYPNISTVMFKFRTFSSSALLMYLATRDLRDFMSVELTDGHIKVSYDLGSGMASVVSNQNHNDGKWKSFTLSRIQKQANISIVDIDTNQEENIATSSSGNNFGLDLKADDKIYFGGLPTLRNLSMKARPEVNLKKYSGCLKDIEISRTPYNILSSPDYVGVTKGCSLENVYTVSFPKPGFVELSPVPIDVGTEINLSFSTKNESGIILLGSGGTPAPPRRKRRQTGQAYYAILLNRGRLEVHLSTGARTMRKIVIRPEPNLFHDGREHSVHVERTRGIFTVQVDENRRYMQNLTVEQPIEVKKLFVGGAPPEFQPSPLRNIPPFEGCIWNLVINSVPMDFARPVSFKNADIGRCAHQKLREDEDGAAPAEIVIQPEPVPTPAFPTPTPVLTHGPCAAESEPALLIGSKQFGLSRNSHIAIAFDDTKVKNRLTIELEVRTEAESGLLFYMARINHADFATVQLRNGLPYFSYDLGSGDTHTMIPTKINDGQWHKIKIMRSKQEGILYVDGASNRTISPKKADILDVVGMLYVGGLPINYTTRRIGPVTYSIDGCVRNLHMAEAPADLEQPTSSFHVGTCFANAQRGTYFDGTGFAKAVGGFKVGLDLLVEFEFRTTTTTGVLLGISSQKMDGMGIEMIDEKLMFHVDNGAGRFTAVYDAGVPGHLCDGQWHKVTANKIKHRIELTVDGNQVEAQSPNPASTSADTNDPVFVGGFPDDLKQFGLTTSIPFRGCIRSLKLTKGTGKPLEVNFAKALELRGVQPVSCPAN",
# GBRR1_HUMAN
"MLAVPNMRFGIFLLWWGWVLATESRMHWPGREVHEMSKKGRPQRQRREVHEDAHKQVSPILRRSPDITKSPLTKSEQLLRIDDHDFSMRPGFGGPAIPVGVDVQVESLDSISEVDMDFTMTLYLRHYWKDERLSFPSTNNLSMTFDGRLVKKIWVPDMFFVHSKRSFIHDTTTDNVMLRVQPDGKVLYSLRVTVTAMCNMDFSRFPLDTQTCSLEIESYAYTEDDLMLYWKKGNDSLKTDERISLSQFLIQEFHTTTKLAFYSSTGWYNRLYINFTLRRHIFFFLLQTYFPATLMVMLSWVSFWIDRRAVPARVPLGITTVLTMSTIITGVNASMPRVSYIKAVDIYLWVSFVFVFLSVLEYAAVNYLTTVQERKEQKLREKLPCTSGLPPPRTAMLDGNYSDGEVNDLDNYMPENGEKPDRMMVQLTLASERSSPQRKSQRSSYVSMRIDTHAIDKYSRIIFPAAYILFNLIYWSIFS",
# AKAP5_HUMAN
"METTISEIHVENKDEKRSAEGSPGAERQKEKASMLCFKRRKKAAKALKPKAGSEAADVARKCPQEAGASDQPEPTRGAWASLKRLVTRRKRSESSKQQKPLEGEMQPAINAEDADLSKKKAKSRLKIPCIKFPRGPKRSNHSKIIEDSDCSIKVQEEAEILDIQTQTPLNDQATKAKSTQDLSEGISRKDGDEVCESNVSNSTTSGEKVISVELGLDNGHSAIQTGTLILEEIETIKEKQDVQPQQASPLETSETDHQQPVLSDVPPLPAIPDQQIVEEASNSTLESAPNGKDYESTEIVAEETKPKDTELSQESDFKENGITEEKSKSEESKRMEPIAIIITDTEISEFDVTKSKNVPKQFLISAENEQVGVFANDNGFEDRTSEQYETLLIETASSLVKNAIQLSIEQLVNEMASDDNKINNLLQ",
# TENA_HUMAN
"MGAMTQLLAGVFLAFLALATEGGVLKKVIRHKRQSGVNATLPEENQPVVFNHVYNIKLPVGSQCSVDLESASGEKDLAPPSEPSESFQEHTVDGENQIVFTHRINIPRRACGCAAAPDVKELLSRLEELENLVSSLREQCTAGAGCCLQPATGRLDTRPFCSGRGNFSTEGCGCVCEPGWKGPNCSEPECPGNCHLRGRCIDGQCICDDGFTGEDCSQLACPSDCNDQGKCVNGVCICFEGYAGADCSREICPVPCSEEHGTCVDGLCVCHDGFAGDDCNKPLCLNNCYNRGRCVENECVCDEGFTGEDCSELICPNDCFDRGRCINGTCYCEEGFTGEDCGKPTCPHACHTQGRCEEGQCVCDEGFAGVDCSEKRCPADCHNRGRCVDGRCECDDGFTGADCGELKCPNGCSGHGRCVNGQCVCDEGYTGEDCSQLRCPNDCHSRGRCVEGKCVCEQGFKGYDCSDMSCPNDCHQHGRCVNGMCVCDDGYTGEDCRDRQCPRDCSNRGLCVDGQCVCEDGFTGPDCAELSCPNDCHGQGRCVNGQCVCHEGFMGKDCKEQRCPSDCHGQGRCVDGQCICHEGFTGLDCGQHSCPSDCNNLGQCVSGRCICNEGYSGEDCSEVSPPKDLVVTEVTEETVNLAWDNEMRVTEYLVVYTPTHEGGLEMQFRVPGDQTSTIIQELEPGVEYFIRVFAILENKKSIPVSARVATYLPAPEGLKFKSIKETSVEVEWDPLDIAFETWEIIFRNMNKEDEGEITKSLRRPETSYRQTGLAPGQEYEISLHIVKNNTRGPGLKRVTTTRLDAPSQIEVKDVTDTTALITWFKPLAEIDGIELTYGIKDVPGDRTTIDLTEDENQYSIGNLKPDTEYEVSLISRRGDMSSNPAKETFTTGLDAPRNLRRVSQTDNSITLEWRNGKAAIDSYRIKYAPISGGDHAEVDVPKSQQATTKTTLTGLRPGTEYGIGVSAVKEDKESNPATINAATELDTPKDLQVSETAETSLTLLWKTPLAKFDRYRLNYSLPTGQWVGVQLPRNTTSYVLRGLEPGQEYNVLLTAEKGRHKSKPARVKASTEQAPELENLTVTEVGWDGLRLNWTAADQAYEHFIIQVQEANKVEAARNLTVPGSLRAVDIPGLKAATPYTVSIYGVIQGYRTPVLSAEASTGETPNLGEVVVAEVGWDALKLNWTAPEGAYEYFFIQVQEADTVEAAQNLTVPGGLRSTDLPGLKAATHYTITIRGVTQDFSTTPLSVEVLTEEVPDMGNLTVTEVSWDALRLNWTTPDGTYDQFTIQVQEADQVEEAHNLTVPGSLRSMEIPGLRAGTPYTVTLHGEVRGHSTRPLAVEVVTEDLPQLGDLAVSEVGWDGLRLNWTAADNAYEHFVIQVQEVNKVEAAQNLTLPGSLRAVDIPGLEAATPYRVSIYGVIRGYRTPVLSAEASTAKEPEIGNLNVSDITPESFNLSWMATDGIFETFTIEIIDSNRLLETVEYNISGAERTAHISGLPPSTDFIVYLSGLAPSIRTKTISATATTEALPLLENLTISDINPYGFTVSWMASENAFDSFLVTVVDSGKLLDPQEFTLSGTQRKLELRGLITGIGYEVMVSGFTQGHQTKPLRAEIVTEAEPEVDNLLVSDATPDGFRLSWTADEGVFDNFVLKIRDTKKQSEPLEITLLAPERTRDITGLREATEYEIELYGISKGRRSQTVSAIATTAMGSPKEVIFSDITENSATVSWRAPTAQVESFRITYVPITGGTPSMVTVDGTKTQTRLVKLIPGVEYLVSIIAMKGFEESEPVSGSFTTALDGPSGLVTANITDSEALARWQPAIATVDSYVISYTGEKVPEITRTVSGNTVEYALTDLEPATEYTLRIFAEKGPQKSSTITAKFTTDLDSPRDLTATEVQSETALLTWRPPRASVTGYLLVYESVDGTVKEVIVGPDTTSYSLADLSPSTHYTAKIQALNGPLRSNMIQTIFTTIGLLYPFPKDCSQAMLNGDTTSGLYTIYLNGDKAEALEVFCDMTSDGGGWIVFLRRKNGRENFYQNWKAYAAGFGDRREEFWLGLDNLNKITAQGQYELRVDLRDHGETAFAVYDKFSVGDAKTRYKLKVEGYSGTAGDSMAYHNGRSFSTFDKDTDSAITNCALSYKGAFWYRNCHRVNLMGRYGDNNHSQGVNWFHWKGHEHSIQFAEMKLRPSNFRNLEGRRKRA",
# APC_HUMAN
"MAAASYDQLLKQVEALKMENSNLRQELEDNSNHLTKLETEASNMKEVLKQLQGSIEDEAMASSGQIDLLERLKELNLDSSNFPGVKLRSKMSLRSYGSREGSVSSRSGECSPVPMGSFPRRGFVNGSRESTGYLEELEKERSLLLADLDKEEKEKDWYYAQLQNLTKRIDSLPLTENFSLQTDMTRRQLEYEARQIRVAMEEQLGTCQDMEKRAQRRIARIQQIEKDILRIRQLLQSQATEAERSSQNKHETGSHDAERQNEGQGVGEINMATSGNGQGSTTRMDHETASVLSSSSTHSAPRRLTSHLGTKVEMVYSLLSMLGTHDKDDMSRTLLAMSSSQDSCISMRQSGCLPLLIQLLHGNDKDSVLLGNSRGSKEARARASAALHNIIHSQPDDKRGRREIRVLHLLEQIRAYCETCWEWQEAHEPGMDQDKNPMPAPVEHQICPAVCVLMKLSFDEEHRHAMNELGGLQAIAELLQVDCEMYGLTNDHYSITLRRYAGMALTNLTFGDVANKATLCSMKGCMRALVAQLKSESEDLQQVIASVLRNLSWRADVNSKKTLREVGSVKALMECALEVKKESTLKSVLSALWNLSAHCTENKADICAVDGALAFLVGTLTYRSQTNTLAIIESGGGILRNVSSLIATNEDHRQILRENNCLQTLLQHLKSHSLTIVSNACGTLWNLSARNPKDQEALWDMGAVSMLKNLIHSKHKMIAMGSAAALRNLMANRPAKYKDANIMSPGSSLPSLHVRKQKALEAELDAQHLSETFDNIDNLSPKASHRSKQRHKQSLYGDYVFDTNRHDDNRSDNFNTGNMTVLSPYLNTTVLPSSSSSRGSLDSSRSEKDRSLERERGIGLGNYHPATENPGTSSKRGLQISTTAAQIAKVMEEVSAIHTSQEDRSSGSTTELHCVTDERNALRRSSAAHTHSNTYNFTKSENSNRTCSMPYAKLEYKRSSNDSLNSVSSSDGYGKRGQMKPSIESYSEDDESKFCSYGQYPADLAHKIHSANHMDDNDGELDTPINYSLKYSDEQLNSGRQSPSQNERWARPKHIIEDEIKQSEQRQSRNQSTTYPVYTESTDDKHLKFQPHFGQQECVSPYRSRGANGSETNRVGSNHGINQNVSQSLCQEDDYEDDKPTNYSERYSEEEQHEEEERPTNYSIKYNEEKRHVDQPIDYSLKYATDIPSSQKQSFSFSKSSSGQSSKTEHMSSSSENTSTPSSNAKRQNQLHPSSAQSRSGQPQKAATCKVSSINQETIQTYCVEDTPICFSRCSSLSSLSSAEDEIGCNQTTQEADSANTLQIAEIKEKIGTRSAEDPVSEVPAVSQHPRTKSSRLQGSSLSSESARHKAVEFSSGAKSPSKSGAQTPKSPPEHYVQETPLMFSRCTSVSSLDSFESRSIASSVQSEPCSGMVSGIISPSDLPDSPGQTMPPSRSKTPPPPPQTAQTKREVPKNKAPTAEKRESGPKQAAVNAAVQRVQVLPDADTLLHFATESTPDGFSCSSSLSALSLDEPFIQKDVELRIMPPVQENDNGNETESEQPKESNENQEKEAEKTIDSEKDLLDDSDDDDIEILEECIISAMPTKSSRKAKKPAQTASKLPPPVARKPSQLPVYKLLPSQNRLQPQKHVSFTPGDDMPRVYCVEGTPINFSTATSLSDLTIESPPNELAAGEGVRGGAQSGEFEKRDTIPTEGRSTDEAQGGKTSSVTIPELDDNKAEEGDILAECINSAMPKGKSHKPFRVKKIMDQVQQASASSSAPNKNQLDGKKKKPTSPVKPIPQNTEYRTRVRKNADSKNNLNAERVFSDNKDSKKQNLKNNSKVFNDKLPNNEDRVRGSFAFDSPHHYTPIEGTPYCFSRNDSLSSLDFDDDDVDLSREKAELRKAKENKESEAKVTSHTELTSNQQSANKTQAIAKQPINRGQPKPILQKQSTFPQSSKDIPDRGAATDEKLQNFAIENTPVCFSHNSSLSSLSDIDQENNNKENEPIKETEPPDSQGEPSKPQASGYAPKSFHVEDTPVCFSRNSSLSSLSIDSEDDLLQECISSAMPKKKKPSRLKGDNEKHSPRNMGGILGEDLTLDLKDIQRPDSEHGLSPDSENFDWKAIQEGANSIVSSLHQAAAAACLSRQASSDSDSILSLKSGISLGSPFHLTPDQEEKPFTSNKGPRILKPGEKSTLETKKIESESKGIKGGKKVYKSLITGKVRSNSEISGQMKQPLQANMPSISRGRTMIHIPGVRNSSSSTSPVSKKGPPLKTPASKSPSEGQTATTSPRGAKPSVKSELSPVARQTSQIGGSSKAPSRSGSRDSTPSRPAQQPLSRPIQSPGRNSISPGRNGISPPNKLSQLPRTSSPSTASTKSSGSGKMSYTSPGRQMSQQNLTKQTGLSKNASSIPRSESASKGLNQMNNGNGANKKVELSRMSSTKSSGSESDRSERPVLVRQSTFIKEAPSPTLRRKLEESASFESLSPSSRPASPTRSQAQTPVLSPSLPDMSLSTHSSVQAGGWRKLPPNLSPTIEYNDGRPAKRHDIARSHSESPSRLPINRSGTWKREHSKHSSSLPRVSTWRRTGSSSSILSASSESSEKAKSEDEKHVNSISGTKQSKENQVSAKGTWRKIKENEFSPTNSTSQTVSSGATNGAESKTLIYQMAPAVSKTEDVWVRIEDCPINNPRSGRSPTGNTPPVIDSVSEKANPNIKDSKDNQAKQNVGNGSVPMRTVGLENRLNSFIQVDAPDQKGTEIKPGQNNPVPVSETNESSIVERTPFSSSSSSKHSSPSGTVAARVTPFNYNPSPRKSSADSTSARPSQIPTPVNNNTKKRDSKTDSTESSGTQSPKRHSGSYLVTSV",
# ARBK1_HUMAN
"MADLEAVLADVSYLMAMEKSKATPAARASKKILLPEPSIRSVMQKYLEDRGEVTFEKIFSQKLGYLLFRDFCLNHLEEARPLVEFYEEIKKYEKLETEEERVARSREIFDSYIMKELLACSHPFSKSATEHVQGHLGKKQVPPDLFQPYIEEICQNLRGDVFQKFIESDKFTRFCQWKNVELNIHLTMNDFSVHRIIGRGGFGEVYGCRKADTGKMYAMKCLDKKRIKMKQGETLALNERIMLSLVSTGDCPFIVCMSYAFHTPDKLSFILDLMNGGDLHYHLSQHGVFSEADMRFYAAEIILGLEHMHNRFVVYRDLKPANILLDEHGHVRISDLGLACDFSKKKPHASVGTHGYMAPEVLQKGVAYDSSADWFSLGCMLFKLLRGHSPFRQHKTKDKHEIDRMTLTMAVELPDSFSPELRSLLEGLLQRDVNRRLGCLGRGAQEVKESPFFRSLDWQMVFLQKYPPPLIPPRGEVNAADAFDIGSFDEEDTKGIKLLDSDQELYRNFPLTISERWQQEVAETVFDTINAETDRLEARKKAKNKQLGHEEDYALGKDCIMHGYMSKMGNPFLTQWQRRYFYLFPNRLEWRGEGEAPQSLLTMEEIQSVEETQIKERKCLLLKIRGGKQFILQCDSDPELVQWKKELRDAYREAQQLVQRVPKMKNKPRSPVVELSKVPLVQRGSANGL",
# ADA1D_HUMAN
"MTFRDLLSVSFEGPRPDSSAGGSSAGGGGGSAGGAAPSEGPAVGGVPGGAGGGGGVVGAGSGEDNRSSAGEPGSAGAGGDVNGTAAVGGLVVSAQGVGVGVFLAAFILMAVAGNLLVILSVACNRHLQTVTNYFIVNLAVADLLLSATVLPFSATMEVLGFWAFGRAFCDVWAAVDVLCCTASILSLCTISVDRYVGVRHSLKYPAIMTERKAAAILALLWVVALVVSVGPLLGWKEPVPPDERFCGITEEAGYAVFSSVCSFYLPMAVIVVMYCRVYVVARSTTRSLEAGVKRERGKASEVVLRIHCRGAATGADGAHGMRSAKGHTFRSSLSVRLLKFSREKKAAKTLAIVVGVFVLCWFPFFFVLPLGSLFPQLKPSEGVFKVIFWLGYFNSCVNPLIYPCSSREFKRAFLRLLRCQCRRRRRRRPLWRVYGHHWRASTSGLRQDCAPSSGDAPPGAPLALTALPDPDPEPPGTPEMQAPVASRRKPPSAFREWRLLGPFRRPTTQLRAKVSSLSHKIRAGGAQRAEAACAQRSEVEAVSLGVPHEVAEGATCQAYELADYSNLRETDI",
# NK1R_HUMAN
"MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTSVVGNVVVMWIILAHKRMRTVTNYFLVNLAFAEASMAAFNTVVNFTYAVHNEWYYGLFYCKFHNFFPIAAVFASIYSMTAVAFDRYMAIIHPLQPRLSATATKVVICVIWVLALLLAFPQGYYSTTETMPSRVVCMIEWPEHPNKIYEKVYHICVTVLIYFLPLLVIGYAYTVVGITLWASEIPGDSSDRYHEQVSAKRKVVKMMIVVVCTFAICWLPFHIFFLLPYINPDLYLKKFIQQVYLAIMWLAMSSTMYNPIIYCCLNDRFRLGFKHAFRCCPFISAGDYEGLEMKSTRYLQTQGSVYKVSRLETTISTVVGAHEEEPEDGPKATPSSLDLTSNCSSRSDSKTMTESFSFSSNVLS",
# RS12_HUMAN
"MAEEGIAAGGVMDVNTALQEVLKTALIHDGLARGIREAAKALDKRQAHLCVLASNCDEPMYVKLVEALCAEHQINLIKVDDNKKLGEWVGLCKIDREGKPRKVVGCSCVVVKDYGKESQAKDVIEEYFKCKK",
# DNJB1_HUMAN
"MGKDYYQTLGLARGASDEEIKRAYRRQALRYHPDKNKEPGAEEKFKEIAEAYDVLSDPRKREIFDRYGEEGLKGSGPSGGSGGGANGTSFSYTFHGDPHAMFAEFFGGRNPFDTFFGQRNGEEGMDIDDPFSGFPMGMGGFTNVNFGRSRSAQEPARKKQDPPVTHDLRVSLEEIYSGCTKKMKISHKRLNPDGKSIRNEDKILTIEVKKGWKEGTKITFPKEGDQTSNNIPADIVFVLKDKPHNIFKRDGSDVIYPARISLREALCGCTVNVPTLDGRTIPVVFKDVIRPGMRRKVPGEGLPLPKTPEKRGDLIIEFEVIFPERIPQTSRTVLEQVLPI",
# NPY1R_HUMAN
"MNSTLFSQVENHSVHSNFSEKNAQLLAFENDDCHLPLAMIFTLALAYGAVIILGVSGNLALIIIILKQKEMRNVTNILIVNLSFSDLLVAIMCLPFTFVYTLMDHWVFGEAMCKLNPFVQCVSITVSIFSLVLIAVERHQLIINPRGWRPNNRHAYVGIAVIWVLAVASSLPFLIYQVMTDEPFQNVTLDAYKDKYVCFDQFPSDSHRLSYTTLLLVLQYFGPLCFIFICYFKIYIRLKRRNNMMDKMRDNKYRSSETKRINIMLLSIVVAFAVCWLPLTIFNTVFDWNHQIIATCNHNLLFLLCHLTAMISTCVNPIFYGFLNKNFQRDLQFFFNFCDFRSRDDDYETIAMSTMHTDVSKTSLKQASPVAFKKINNNDDNEKI",
# ITA3_HUMAN
"MGPGPSRAPRAPRLMLCALALMVAAGGCVVSAFNLDTRFLVVKEAGNPGSLFGYSVALHRQTERQQRYLLLAGAPRELAVPDGYTNRTGAVYLCPLTAHKDDCERMNITVKNDPGHHIIEDMWLGVTVASQGPAGRVLVCAHRYTQVLWSGSEDQRRMVGKCYVRGNDLELDSSDDWQTYHNEMCNSNTDYLETGMCQLGTSGGFTQNTVYFGAPGAYNWKGNSYMIQRKEWDLSEYSYKDPEDQGNLYIGYTMQVGSFILHPKNITIVTGAPRHRHMGAVFLLSQEAGGDLRRRQVLEGSQVGAYFGSAIALADLNNDGWQDLLVGAPYYFERKEEVGGAIYVFMNQAGTSFPAHPSLLLHGPSGSAFGLSVASIGDINQDGFQDIAVGAPFEGLGKVYIYHSSSKGLLRQPQQVIHGEKLGLPGLATFGYSLSGQMDVDENFYPDLLVGSLSDHIVLLRARPVINIVHKTLVPRPAVLDPALCTATSCVQVELCFAYNQSAGNPNYRRNITLAYTLEADRDRRPPRLRFAGSESAVFHGFFSMPEMRCQKLELLLMDNLRDKLRPIIISMNYSLPLRMPDRPRLGLRSLDAYPILNQAQALENHTEVQFQKECGPDNKCESNLQMRAAFVSEQQQKLSRLQYSRDVRKLLLSINVTNTRTSERSGEDAHEALLTLVVPPALLLSSVRPPGACQANETIFCELGNPFKRNQRMELLIAFEVIGVTLHTRDLQVQLQLSTSSHQDNLWPMILTLLVDYTLQTSLSMVNHRLQSFFGGTVMGESGMKTVEDVGSPLKYEFQVGPMGEGLVGLGTLVLGLEWPYEVSNGKWLLYPTEITVHGNGSWPCRPPGDLINPLNLTLSDPGDRPSSPQRRRRQLDPGGGQGPPPVTLAAAKKAKSETVLTCATGRAHCVWLECPIPDAPVVTNVTVKARVWNSTFIEDYRDFDRVRVNGWATLFLRTSIPTINMENKTTWFSVDIDSELVEELPAEIELWLVLVAVGAGLLLLGLIILLLWKCGFFKRARTRALYEAKRQKAEMKSQPSETERLTDDY",
# CTNA2_HUMAN
"MTSATSPIILKWDPKSLEIRTLTVERLLEPLVTQVTTLVNTSNKGPSGKKKGRSKKAHVLAASVEQATQNFLEKGEQIAKESQDLKEELVAAVEDVRKQGETMRIASSEFADDPCSSVKRGTMVRAARALLSAVTRLLILADMADVMRLLSHLKIVEEALEAVKNATNEQDLANRFKEFGKEMVKLNYVAARRQQELKDPHCRDEMAAARGALKKNATMLYTASQAFLRHPDVAATRANRDYVFKQVQEAIAGISNAAQATSPTDEAKGHTGIGELAAALNEFDNKIILDPMTFSEARFRPSLEERLESIISGAALMADSSCTRDDRRERIVAECNAVRQALQDLLSEYMNNTGRKEKGDPLNIAIDKMTKKTRDLRRQLRKAVMDHISDSFLETNVPLLVLIEAAKSGNEKEVKEYAQVFREHANKLVEVANLACSISNNEEGVKLVRMAATQIDSLCPQVINAALTLAARPQSKVAQDNMDVFKDQWEKQVRVLTEAVDDITSVDDFLSVSENHILEDVNKCVIALQEGDVDTLDRTAGAIRGRAARVIHIINAEMENYEAGVYTEKVLEATKLLSETVMPRFAEQVEVAIEALSANVPQPFEENEFIDASRLVYDGVRDIRKAVLMIRTPEELEDDSDFEQEDYDVRSRTSVQTEDDQLIAGQSARAIMAQLPQEEKAKIAEQVEIFHQEKSKLDAEVAKWDDSGNDIIVLAKQMCMIMMEMTDFTRGKGPLKNTSDVINAAKKIAEAGSRMDKLARAVADQCPDSACKQDLLAYLQRIALYCHQLNICSKVKAEVQNLGGELIVSGTGVQSTFTTFYEVDCDVIDGGRASQLSTHLPTCAEGAPIGSGSSDSSMLDSATSLIQAAKNLMNAVVLTVKASYVASTKYQKVYGTAAVNSPVVSWKMKAPEKKPLVKREKPEEFQTRVRRGSQKKHISPVQALSEFKAMDSF",
# RL13_HUMAN
"MAPSRNGMVLKPHFHKDWQRRVATWFNQPARKIRRRKARQAKARRIAPRPASGPIRPIVRCPTVRYHTKVRAGRGFSLEELRVAGIHKKVARTIGISVDPRRRNKSTESLQANVQRLKEYRSKLILFPRKPSAPKKGDSSAEELKLATQLTGPVMPVRNVYKKEKARVITEEEKNFKAFASLRMARANARLFGIRAKRAKEAAEQDVEKKK",
# ELAV4_HUMAN
"MEWNGLKMIISTMEPQVSNGPTSNTSNGPSSNNRNCPSPMQTGATTDDSKTNLIVNYLPQNMTQEEFRSLFGSIGEIESCKLVRDKITGQSLGYGFVNYIDPKDAEKAINTLNGLRLQTKTIKVSYARPSSASIRDANLYVSGLPKTMTQKELEQLFSQYGRIITSRILVDQVTGVSRGVGFIRFDKRIEAEEAIKGLNGQKPSGATEPITVKFANNPSQKSSQALLSQLYQSPNRRYPGPLHHQAQRFRLDNLLNMAYGVKRLMSGPVPPSACPPRFSPITIDGMTSLVGMNIPGHTGTGWCIFVYNLSPDSDESVLWQLFGPFGAVNNVKVIRDFNTNKCKGFGFVTMTNYDEAAMAIASLNGYRLGDRVLQVSFKTNKAHKS",
# 1433T_HUMAN
"MEKTELIQKAKLAEQAERYDDMATCMKAVTEQGAELSNEERNLLSVAYKNVVGGRRSAWRVISSIEQKTDTSDKKLQLIKDYREKVESELRSICTTVLELLDKYLIANATNPESKVFYLKMKGDYFRYLAEVACGDDRKQTIDNSQGAYQEAFDISKKEMQPTHPIRLGLALNFSVFYYEILNNPELACTLAKTAFDEAIAELDTLNEDSYKDSTLIMQLLRDNLTLWTSDSAGEECDAAEGAEN",
# MK03_HUMAN
"MAAAAAQGGGGGEPRRTEGVGPGVPGEVEMVKGQPFDVGPRYTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLREIQILLRFRHENVIGIRDILRASTLEAMRDVYIVQDLMETDLYKLLKSQQLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLINTTCDLKICDFGLARIADPEHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINMKARNYLQSLPSKTKVAWAKLFPKSDSKALDLLDRMLTFNPNKRITVEEALAHPYLEQYYDPTDEPVAEEPFTFAMELDDLPKERLKELIFQETARFQPGVLEAP",
# VATL_HUMAN
"MSESKSGPEYASFFAVMGASAAMVFSALGAAYGTAKSGTGIAAMSVMRPEQIMKSIIPVVMAGIIAIYGLVVAVLIANSLNDDISLYKSFLQLGAGLSVGLSGLAAGFAIGIVGDAGVRGTAQQPRLFVGMILILIFAEVLGLYGLIVALILSTK",
# RL10_HUMAN
"MGRRPARCYRYCKNKPYPKSRFCRGVPDAKIRIFDLGRKKAKVDEFPLCGHMVSDEYEQLSSEALEAARICANKYMVKSCGKDGFHIRVRLHPFHVIRINKMLSCAGADRLQTGMRGAFGKPQGTVARVHIGQVIMSIRTKLQNKEHVIEALRRAKFKFPGRQKIHISKKWGFTKFNADEFEDMVAEKRLIPDGCGVKYIPSRGPLDKWRALHS",
# APEX1_HUMAN
"MPKRGKKGAVAEDGDELRTEPEAKKSKTAAKKNDKEAAGEGPALYEDPPDQKTSPSGKPATLKICSWNVDGLRAWIKKKGLDWVKEEAPDILCLQETKCSENKLPAELQELPGLSHQYWSAPSDKEGYSGVGLLSRQCPLKVSYGIGDEEHDQEGRVIVAEFDSFVLVTAYVPNAGRGLVRLEYRQRWDEAFRKFLKGLASRKPLVLCGDLNVAHEEIDLRNPKGNKKNAGFTPQERQGFGELLQAVPLADSFRHLYPNTPYAYTFWTYMMNARSKNVGWRLDYFLLSHSLLPALCDSKIRSKALGSDHCPITLYLAL",
# CALR_HUMAN
"MLLSVPLLLGLLGLAVAEPAVYFKEQFLDGDGWTSRWIESKHKSDFGKFVLSSGKFYGDEEKDKGLQTSQDARFYALSASFEPFSNKGQTLVVQFTVKHEQNIDCGGGYVKLFPNSLDQTDMHGDSEYNIMFGPDICGPGTKKVHVIFNYKGKNVLINKDIRCKDDEFTHLYTLIVRPDNTYEVKIDNSQVESGSLEDDWDFLPPKKIKDPDASKPEDWDERAKIDDPTDSKPEDWDKPEHIPDPDAKKPEDWDEEMDGEWEPPVIQNPEYKGEWKPRQIDNPDYKGTWIHPEIDNPEYSPDPSIYAYDNFGVLGLDLWQVKSGTIFDNFLITNDEAYAEEFGNETWGVTKAAEKQMKDKQDEEQRLKEEEEDKKRKEEEEAEDKEDDEDKDEDEEDEEDKEEDEEEDVPGQAKDEL",
# PDE4A_HUMAN
"MEPPTVPSERSLSLSLPGPREGQATLKPPPQHLWRQPRTPIRIQQRGYSDSAERAERERQPHRPIERADAMDTSDRPGLRTTRMSWPSSFHGTGTGSGGAGGGSSRRFEAENGPTPSPGRSPLDSQASPGLVLHAGAATSQRRESFLYRSDSDYDMSPKTMSRNSSVTSEAHAEDLIVTPFAQVLASLRSVRSNFSLLTNVPVPSNKRSPLGGPTPVCKATLSEETCQQLARETLEELDWCLEQLETMQTYRSVSEMASHKFKRMLNRELTHLSEMSRSGNQVSEYISTTFLDKQNEVEIPSPTMKEREKQQAPRPRPSQPPPPPVPHLQPMSQITGLKKLMHSNSLNNSNIPRFGVKTDQEELLAQELENLNKWGLNIFCVSDYAGGRSLTCIMYMIFQERDLLKKFRIPVDTMVTYMLTLEDHYHADVAYHNSLHAADVLQSTHVLLATPALDAVFTDLEILAALFAAAIHDVDHPGVSNQFLINTNSELALMYNDESVLENHHLAVGFKLLQEDNCDIFQNLSKRQRQSLRKMVIDMVLATDMSKHMTLLADLKTMVETKKVTSSGVLLLDNYSDRIQVLRNMVHCADLSNPTKPLELYRQWTDRIMAEFFQQGDRERERGMEISPMCDKHTASVEKSQVGFIDYIVHPLWETWADLVHPDAQEILDTLEDNRDWYYSAIRQSPSPPPEEESRGPGHPPLPDKFQFELTLEEEEEEEISMAQIPCTAQEALTAQGLSGVEEALDATIAWEASPAQESLEVMAQEASLEAELEAVYLTQQAQSTGSAPVAPDEFSSREEFVVAVSHSSPSALALQSPLLPAWRTLSVSEHAPGLPGLPSTAAEVEAQREHQAAKRACSACAGTFGEDTSALPAPGGGGSGGDPT",
# CALX_HUMAN
"MEGKWLLCMLLVLGTAIVEAHDGHDDDVIDIEDDLDDVIEEVEDSKPDTTAPPSSPKVTYKAPVPTGEVYFADSFDRGTLSGWILSKAKKDDTDDEIAKYDGKWEVEEMKESKLPGDKGLVLMSRAKHHAISAKLNKPFLFDTKPLIVQYEVNFQNGIECGGAYVKLLSKTPELNLDQFHDKTPYTIMFGPDKCGEDYKLHFIFRHKNPKTGIYEEKHAKRPDADLKTYFTDKKTHLYTLILNPDNSFEILVDQSVVNSGNLLNDMTPPVNPSREIEDPEDRKPEDWDERPKIPDPEAVKPDDWDEDAPAKIPDEEATKPEGWLDDEPEYVPDPDAEKPEDWDEDMDGEWEAPQIANPRCESAPGCGVWQRPVIDNPNYKGKWKPPMIDNPSYQGIWKPRKIPNPDFFEDLEPFRMTPFSAIGLELWSMTSDIFFDNFIICADRRIVDDWANDGWGLKKAADGAAEPGVVGQMIEAAEERPWLWVVYILTVALPVFLVILFCCSGKKQTSGMEYKKTDAPQPDVKEEEEEKEEEKDKGDEEEEGEEKLEEKQKSDAEEDGGTVSQEEEDRKPKAEEDEILNRSPRNRKPRRE",
# 5HT1D_HUMAN
"MSPLNQSAEGLPQEASNRSLNATETSEAWDPRTLQALKISLAVVLSVITLATVLSNAFVLTTILLTRKLHTPANYLIGSLATTDLLVSILVMPISIAYTITHTWNFGQILCDIWLSSDITCCTASILHLCVIALDRYWAITDALEYSKRRTAGHAATMIAIVWAISICISIPPLFWRQAKAQEEMSDCLVNTSQISYTIYSTCGAFYIPSVLLIILYGRIYRAARNRILNPPSLYGKRFTTAHLITGSAGSSLCSLNSSLHEGHSHSAGSPLFFNHVKIKLADSALERKRISAARERKATKILGIILGAFIICWLPFFVVSLVLPICRDSCWIHPALFDFFTWLGYLNSLINPIIYTVFNEEFRQAFQKIVPFRKAS",
# 5HT1B_HUMAN
"MEEPGAQCAPPPPAGSETWVPQANLSSAPSQNCSAKDYIYQDSISLPWKVLLVMLLALITLATTLSNAFVIATVYRTRKLHTPANYLIASLAVTDLLVSILVMPISTMYTVTGRWTLGQVVCDFWLSSDITCCTASILHLCVIALDRYWAITDAVEYSAKRTPKRAAVMIALVWVFSISISLPPFFWRQAKAEEEVSECVVNTDHILYTVYSTVGAFYFPTLLLIALYGRIYVEARSRILKQTPNRTGKRLTRAQLITDSPGSTSSVTSINSRVPDVPSESGSPVYVNQVKVRVSDALLEKKKLMAARERKATKTLGIILGAFIVCWLPFFIISLVMPICKDACWFHLAIFDFFTWLGYLNSLINPIIYTMSNEDFKQAFHKLIRFKCTS",
# 5HT2A_HUMAN
"MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWTVDSENRTNLSCEGCLSPSCLSLLHLQEKNWSALLTAVVIILTIAGNILVIMAVSLEKKLQNATNYFLMSLAIADMLLGFLVMPVSMLTILYGYRWPLPSKLCAVWIYLDVLFSTASIMHLCAISLDRYVAIQNPIHHSRFNSRTKAFLKIIAVWTISVGISMPIPVFGLQDDSKVFKEGSCLLADDNFVLIGSFVSFFIPLTIMVITYFLTIKSLQKEATLCVSDLGTRAKLASFSFLPQSSLSSEKLFQRSIHREPGSYTGRRTMQSISNEQKACKVLGIVFFLFVVMWCPFFITNIMAVICKESCNEDVIGALLNVFVWIGYLSSAVNPLVYTLFNKTYRSAFSRYIQCQYKENKKPLQLILVNTIPALAYKSSQLQMGQKKNSKQDAKTTDNDCSMVALGKQHSEEASKDNSDGVNEKVSCV",
# GBRB3_HUMAN
"MWGLAGGRLFGIFSAPVLVAVVCCAQSVNDPGNMSFVKETVDKLLKGYDIRLRPDFGGPPVCVGMNIDIASIDMVSEVNMDYTLTMYFQQYWRDKRLAYSGIPLNLTLDNRVADQLWVPDTYFLNDKKSFVHGVTVKNRMIRLHPDGTVLYGLRITTTAACMMDLRRYPLDEQNCTLEIESYGYTTDDIEFYWRGGDKAVTGVERIELPQFSIVEHRLVSRNVVFATGAYPRLSLSFRLKRNIGYFILQTYMPSILITILSWVSFWINYDASAARVALGITTVLTMTTINTHLRETLPKIPYVKAIDMYLMGCFVFVFLALLEYAFVNYIFFGRGPQRQKKLAEKTAKAKNDRSKSESNRVDAHGNILLTSLEVHNEMNEVSGGIGDTRNSAISFDNSGIQYRKQSMPREGHGRFLGDRSLPHKKTHLRRRSSQLKIKIPDLTDVNAIDRWSRIVFPFTFSLFNLVYWLYYVN",
# GBRR2_HUMAN
"MPYFTRLILFLFCLMVLVESRKPKRKRWTGQVEMPKPSHLYKKNLDVTKIRKGKPQQLLRVDEHDFSMRPAFGGPAIPVGVDVQVESLDSISEVDMDFTMTLYLRHYWKDERLAFSSASNKSMTFDGRLVKKIWVPDVFFVHSKRSFTHDTTTDNIMLRVFPDGHVLYSMRITVTAMCNMDFSHFPLDSQTCSLELESYAYTDEDLMLYWKNGDESLKTDEKISLSQFLIQKFHTTSRLAFYSSTGWYNRLYINFTLRRHIFFFLLQTYFPATLMVMLSWVSFWIDRRAVPARVSLGITTVLTMTTIITGVNASMPRVSYVKAVDIYLWVSFVFVFLSVLEYAAVNYLTTVQERKERKLREKFPCMCGMLHSKTMMLDGSYSESEANSLAGYPRSHILTEEERQDKIVVHLGLSGEANAARKKGLLKGQTGFRIFQNTHAIDKYSRLIFPASYIFFNLIYWSVFS",
# MK01_HUMAN
"MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNVNKVRVAIKKISPFEHQTYCQRTLREIKILLRFRHENIIGINDIIRAPTIEQMKDVYIVQDLMETDLYKLLKTQHLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLLNTTCDLKICDFGLARVADPDHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINLKARNYLLSLPHKNKVPWNRLFPNADSKALDLLDKMLTFNPHKRIEVEQALAHPYLEQYYDPSDEPIAEAPFKFDMELDDLPKEKLKELIFEETARFQPGYRS",
# GRN_HUMAN
"MWTLVSWVALTAGLVAGTRCPDGQFCPVACCLDPGGASYSCCRPLLDKWPTTLSRHLGGPCQVDAHCSAGHSCIFTVSGTSSCCPFPEAVACGDGHHCCPRGFHCSADGRSCFQRSGNNSVGAIQCPDSQFECPDFSTCCVMVDGSWGCCPMPQASCCEDRVHCCPHGAFCDLVHTRCITPTGTHPLAKKLPAQRTNRAVALSSSVMCPDARSRCPDGSTCCELPSGKYGCCPMPNATCCSDHLHCCPQDTVCDLIQSKCLSKENATTDLLTKLPAHTVGDVKCDMEVSCPDGYTCCRLQSGAWGCCPFTQAVCCEDHIHCCPAGFTCDTQKGTCEQGPHQVPWMEKAPAHLSLPDPQALKRDVPCDNVSSCPSSDTCCQLTSGEWGCCPIPEAVCCSDHQHCCPQGYTCVAEGQCQRGSEIVAGLEKMPARRASLSHPRDIGCDQHTSCPVGQTCCPSLGGSWACCQLPHAVCCEDRQHCCPAGYTCNVKARSCEKEVVSAQPATFLARSPHVGVKDVECGEGHFCHDNQTCCRDNRQGWACCPYRQGVCCADRRHCCPAGFRCAARGTKCLRREAPRWDAPLRDPALRQLL",
# AA2AR_HUMAN
"MPIMGSSVYITVELAIAVLAILGNVLVCWAVWLNSNLQNVTNYFVVSLAAADIAVGVLAIPFAITISTGFCAACHGCLFIACFVLVLTQSSIFSLLAIAIDRYIAIRIPLRYNGLVTGTRAKGIIAICWVLSFAIGLTPMLGWNNCGQPKEGKNHSQGCGEGQVACLFEDVVPMNYMVYFNFFACVLVPLLLMLGVYLRIFLAARRQLKQMESQPLPGERARSTLQKEVHAAKSLAIIVGLFALCWLPLHIINCFTFFCPDCSHAPLWLMYLAIVLSHTNSVVNPFIYAYRIREFRQTFRKIIRSHVLRQQEPFKAAGTSARVLAAHGSDGEQVSLRLNGHPPGVWANGSAPHPERRPNGYALGLVSGGSAQESQGNTGLPDVELLSHELKGVCPEPPGLDDPLAQDGAGVS",
# AA2BR_HUMAN
"MLLETQDALYVALELVIAALSVAGNVLVCAAVGTANTLQTPTNYFLVSLAAADVAVGLFAIPFAITISLGFCTDFYGCLFLACFVLVLTQSSIFSLLAVAVDRYLAICVPLRYKSLVTGTRARGVIAVLWVLAFGIGLTPFLGWNSKDSATNNCTEPWDGTTNESCCLVKCLFENVVPMSYMVYFNFFGCVLPPLLIMLVIYIKIFLVACRQLQRTELMDHSRTTLQREIHAAKSLAMIVGIFALCWLPVHAVNCVTLFQPAQGKNKPKWAMNMAILLSHANSVVNPIVYAYRNRDFRYTFHKIISRYLLCQADVKSGNGQAGVQPALGVGL",
# EPHA3_HUMAN
"MDCQLSILLLLSCSVLDSFGELIPQPSNEVNLLDSKTIQGELGWISYPSHGWEEISGVDEHYTPIRTYQVCNVMDHSQNNWLRTNWVPRNSAQKIYVELKFTLRDCNSIPLVLGTCKETFNLYYMESDDDHGVKFREHQFTKIDTIAADESFTQMDLGDRILKLNTEIREVGPVNKKGFYLAFQDVGACVALVSVRVYFKKCPFTVKNLAMFPDTVPMDSQSLVEVRGSCVNNSKEEDPPRMYCSTEGEWLVPIGKCSCNAGYEERGFMCQACRPGFYKALDGNMKCAKCPPHSSTQEDGSMNCRCENNYFRADKDPPSMACTRPPSSPRNVISNINETSVILDWSWPLDTGGRKDVTFNIICKKCGWNIKQCEPCSPNVRFLPRQFGLTNTTVTVTDLLAHTNYTFEIDAVNGVSELSSPPRQFAAVSITTNQAAPSPVLTIKKDRTSRNSISLSWQEPEHPNGIILDYEVKYYEKQEQETSYTILRARGTNVTISSLKPDTIYVFQIRARTAAGYGTNSRKFEFETSPDSFSISGESSQVVMIAISAAVAIILLTVVIYVLIGRFCGYKSKHGADEKRLHFGNGHLKLPGLRTYVDPHTYEDPTQAVHEFAKELDATNISIDKVVGAGEFGEVCSGRLKLPSKKEISVAIKTLKVGYTEKQRRDFLGEASIMGQFDHPNIIRLEGVVTKSKPVMIVTEYMENGSLDSFLRKHDAQFTVIQLVGMLRGIASGMKYLSDMGYVHRDLAARNILINSNLVCKVSDFGLSRVLEDDPEAAYTTRGGKIPIRWTSPEAIAYRKFTSASDVWSYGIVLWEVMSYGERPYWEMSNQDVIKAVDEGYRLPPPMDCPAALYQLMLDCWQKDRNNRPKFEQIVSILDKLIRNPGSLKIITSAAARPSNLLLDQSNVDITTFRTTGDWLNGVWTAHCKEIFTGVEYSSCDTIAKISTDDMKKVGVTVVGPQKKIISSIKALETQSKNGPVPV",
# EPHB2_HUMAN
"MALRRLGAALLLLPLLAAVEETLMDSTTATAELGWMVHPPSGWEEVSGYDENMNTIRTYQVCNVFESSQNNWLRTKFIRRRGAHRIHVEMKFSVRDCSSIPSVPGSCKETFNLYYYEADFDSATKTFPNWMENPWVKVDTIAADESFSQVDLGGRVMKINTEVRSFGPVSRSGFYLAFQDYGGCMSLIAVRVFYRKCPRIIQNGAIFQETLSGAESTSLVAARGSCIANAEEVDVPIKLYCNGDGEWLVPIGRCMCKAGFEAVENGTVCRGCPSGTFKANQGDEACTHCPINSRTTSEGATNCVCRNGYYRADLDPLDMPCTTIPSAPQAVISSVNETSLMLEWTPPRDSGGREDLVYNIICKSCGSGRGACTRCGDNVQYAPRQLGLTEPRIYISDLLAHTQYTFEIQAVNGVTDQSPFSPQFASVNITTNQAAPSAVSIMHQVSRTVDSITLSWSQPDQPNGVILDYELQYYEKELSEYNATAIKSPTNTVTVQGLKAGAIYVFQVRARTVAGYGRYSGKMYFQTMTEAEYQTSIQEKLPLIIGSSAAGLVFLIAVVVIAIVCNRRGFERADSEYTDKLQHYTSGHMTPGMKIYIDPFTYEDPNEAVREFAKEIDISCVKIEQVIGAGEFGEVCSGHLKLPGKREIFVAIKTLKSGYTEKQRRDFLSEASIMGQFDHPNVIHLEGVVTKSTPVMIITEFMENGSLDSFLRQNDGQFTVIQLVGMLRGIAAGMKYLADMNYVHRDLAARNILVNSNLVCKVSDFGLSRFLEDDTSDPTYTSALGGKIPIRWTAPEAIQYRKFTSASDVWSYGIVMWEVMSYGERPYWDMTNQDVINAIEQDYRLPPPMDCPSALHQLMLDCWQKDRNHRPKFGQIVNTLDKMIRNPNSLKAMAPLSSGINLPLLDRTIPDYTSFNTVDEWLEAIKMGQYKESFANAGFTSFDVVSQMMMEDILRVGVTLAGHQKKILNSIQVMRAQMNQIQSVEGQPLARRPRATGRTKRCQPRDVTKKTCNSNDGKKKGMGKKKTDPGRGREIQGIFFKEDSHKESNDCSCGG",
# NOS1_HUMAN
"MEDHMFGVQQIQPNVISVRLFKRKVGGLGFLVKERVSKPPVIISDLIRGGAAEQSGLIQAGDIILAVNGRPLVDLSYDSALEVLRGIASETHVVLILRGPEGFTTHLETTFTGDGTPKTIRVTQPLGPPTKAVDLSHQPPAGKEQPLAVDGASGPGNGPQHAYDDGQEAGSLPHANGLAPRPPGQDPAKKATRVSLQGRGENNELLKEIEPVLSLLTSGSRGVKGGAPAKAEMKDMGIQVDRDLDGKSHKPLPLGVENDRVFNDLWGKGNVPVVLNNPYSEKEQPPTSGKQSPTKNGSPSKCPRFLKVKNWETEVVLTDTLHLKSTLETGCTEYICMGSIMHPSQHARRPEDVRTKGQLFPLAKEFIDQYYSSIKRFGSKAHMERLEEVNKEIDTTSTYQLKDTELIYGAKHAWRNASRCVGRIQWSKLQVFDARDCTTAHGMFNYICNHVKYATNKGNLRSAITIFPQRTDGKHDFRVWNSQLIRYAGYKQPDGSTLGDPANVQFTEICIQQGWKPPRGRFDVLPLLLQANGNDPELFQIPPELVLEVPIRHPKFEWFKDLGLKWYGLPAVSNMLLEIGGLEFSACPFSGWYMGTEIGVRDYCDNSRYNILEEVAKKMNLDMRKTSSLWKDQALVEINIAVLYSFQSDKVTIVDHHSATESFIKHMENEYRCRGGCPADWVWIVPPMSGSITPVFHQEMLNYRLTPSFEYQPDPWNTHVWKGTNGTPTKRRAIGFKKLAEAVKFSAKLMGQAMAKRVKATILYATETGKSQAYAKTLCEIFKHAFDAKVMSMEEYDIVHLEHETLVLVVTSTFGNGDPPENGEKFGCALMEMRHPNSVQEERKSYKVRFNSVSSYSDSQKSSGDGPDLRDNFESAGPLANVRFSVFGLGSRAYPHFCAFGHAVDTLLEELGGERILKMREGDELCGQEEAFRTWAKKVFKAACDVFCVGDDVNIEKANNSLISNDRSWKRNKFRLTFVAEAPELTQGLSNVHKKRVSAARLLSRQNLQSPKSSRSTIFVRLHTNGSQELQYQPGDHLGVFPGNHEDLVNALIERLEDAPPVNQMVKVELLEERNTALGVISNWTDELRLPPCTIFQAFKYYLDITTPPTPLQLQQFASLATSEKEKQRLLVLSKGLQEYEEWKWGKNPTIVEVLEEFPSIQMPATLLLTQLSLLQPRYYSISSSPDMYPDEVHLTVAIVSYRTRDGEGPIHHGVCSSWLNRIQADELVPCFVRGAPSFHLPRNPQVPCILVGPGTGIAPFRSFWQQRQFDIQHKGMNPCPMVLVFGCRQSKIDHIYREETLQAKNKGVFRELYTAYSREPDKPKKYVQDILQEQLAESVYRALKEQGGHIYVCGDVTMAADVLKAIQRIMTQQGKLSAEDAGVFISRMRDDNRYHEDIFGVTLRTYEVTNRLRSESIAFIEESKKDTDEVFSS",
# EF1D_HUMAN
"MATNFLAHEKIWFDKFKYDDAERRFYEQMNGPVAGASRQENGASVILRDIARARENIQKSLAGSSGPGASSGTSGDHGELVVRIASLEVENQSLRGVVQELQQAISKLEARLNVLEKSSPGHRATAPQTQHVSPMRQVEPPAKKPATPAEDDEDDDIDLFGSDNEEEDKEAAQLREERLRQYAEKKAKKPALVAKSSILLDVKPWDDETDMAQLEACVRSIQLDGLVWGASKLVPVGYGIRKLQIQCVVEDDKVGTDLLEEEITKFEEHVQSVDIAAFNKI",
# MARCS_HUMAN
"MGAQFSKTAAKGEAAAERPGEAAVASSPSKANGQENGHVKVNGDASPAAAESGAKEELQANGSAPAADKEEPAAAGSGAASPSAAEKGEPAAAAAPEAGASPVEKEAPAEGEAAEPGSPTAAEGEAASAASSTSSPKAEDGATPSPSNETPKKKKKRFSFKKSFKLSGFSFKKNKKEAGEGGEAEAPAAEGGKDEAAGGAAAAAAEAGAASGEQAAAPGEEAAAGEEGAAGGDPQEAKPQEAAVAPEKPPASDETKAAEEPSKVEEKKAEEAGASAAACEAPSAAGPGAPPEQEAAPAEEPAAAAASSACAAPSQEAQPECSPEAPPAEAAE",
# GNA11_HUMAN
"MTLESMMACCLSDEVKESKRINAEIEKQLRRDKRDARRELKLLLLGTGESGKSTFIKQMRIIHGAGYSEEDKRGFTKLVYQNIFTAMQAMIRAMETLKILYKYEQNKANALLIREVDVEKVTTFEHQYVSAIKTLWEDPGIQECYDRRREYQLSDSAKYYLTDVDRIATLGYLPTQQDVLRVRVPTTGIIEYPFDLENIIFRMVDVGGQRSERRKWIHCFENVTSIMFLVALSEYDQVLVESDNENRMEESKALFRTIITYPWFQNSSVILFLNKKDLLEDKILYSHLVDYFPEFDGPQRDAQAAREFILKMFVDLNPDSDKIIYSHFTCATDTENIRFVFAAVKDTILQLNLKEYNLV",
# RL12_HUMAN
"MPPKFDPNEIKVVYLRCTGGEVGATSALAPKIGPLGLSPKKVGDDIAKATGDWKGLRITVKLTIQNRQAQIEVVPSASALIIKALKEPPRDRKKQKNIKHSGNITFDEIVNIARQMRHRSLARELSGTIKEILGTAQSVGCNVDGRHPHDIIDDINSGAVECPAS",
# 2AAA_HUMAN
"MAAADGDDSLYPIAVLIDELRNEDVQLRLNSIKKLSTIALALGVERTRSELLPFLTDTIYDEDEVLLALAEQLGTFTTLVGGPEYVHCLLPPLESLATVEETVVRDKAVESLRAISHEHSPSDLEAHFVPLVKRLAGGDWFTSRTSACGLFSVCYPRVSSAVKAELRQYFRNLCSDDTPMVRRAAASKLGEFAKVLELDNVKSEIIPMFSNLASDEQDSVRLLAVEACVNIAQLLPQEDLEALVMPTLRQAAEDKSWRVRYMVADKFTELQKAVGPEITKTDLVPAFQNLMKDCEAEVRAAASHKVKEFCENLSADCRENVIMSQILPCIKELVSDANQHVKSALASVIMGLSPILGKDNTIEHLLPLFLAQLKDECPEVRLNIISNLDCVNEVIGIRQLSQSLLPAIVELAEDAKWRVRLAIIEYMPLLAGQLGVEFFDEKLNSLCMAWLVDHVYAIREAATSNLKKLVEKFGKEWAHATIIPKVLAMSGDPNYLHRMTTLFCINVLSEVCGQDITTKHMLPTVLRMAGDPVANVRFNVAKSLQKIGPILDNSTLQSEVKPILEKLTQDQDVDVKYFAQEALTVLSLA",
# 2AAB_HUMAN
"MAGASELGTGPGAAGGDGDDSLYPIAVLIDELRNEDVQLRLNSIKKLSTIALALGVERTRSELLPFLTDTIYDEDEVLLALAEQLGNFTGLVGGPDFAHCLLPPLENLATVEETVVRDKAVESLRQISQEHTPVALEAYFVPLVKRLASGDWFTSRTSACGLFSVCYPRASNAVKAEIRQQFRSLCSDDTPMVRRAAASKLGEFAKVLELDSVKSEIVPLFTSLASDEQDSVRLLAVEACVSIAQLLSQDDLETLVMPTLRQAAEDKSWRVRYMVADRFSELQKAMGPKITLNDLIPAFQNLLKDCEAEVRAAAAHKVKELGENLPIEDRETIIMNQILPYIKELVSDTNQHVKSALASVIMGLSTILGKENTIEHLLPLFLAQLKDECPDVRLNIISNLDCVNEVIGIRQLSQSLLPAIVELAEDAKWRVRLAIIEYMPLLAGQLGVEFFDEKLNSLCMAWLVDHVYAIREAATNNLMKLVQKFGTEWAQNTIVPKVLVMANDPNYLHRMTTLFCINALSEACGQEITTKQMLPIVLKMAGDQVANVRFNVAKSLQKIGPILDTNALQGEVKPVLQKLGQDEDMDVKYFAQEAISVLALA",
# SC6A1_HUMAN
"MATNGSKVADGQISTEVSEAPVANDKPKTLVVKVQKKAADLPDRDTWKGRFDFLMSCVGYAIGLGNVWRFPYLCGKNGGGAFLIPYFLTLIFAGVPLFLLECSLGQYTSIGGLGVWKLAPMFKGVGLAAAVLSFWLNIYYIVIISWAIYYLYNSFTTTLPWKQCDNPWNTDRCFSNYSMVNTTNMTSAVVEFWERNMHQMTDGLDKPGQIRWPLAITLAIAWILVYFCIWKGVGWTGKVVYFSATYPYIMLIILFFRGVTLPGAKEGILFYITPNFRKLSDSEVWLDAATQIFFSYGLGLGSLIALGSYNSFHNNVYRDSIIVCCINSCTSMFAGFVIFSIVGFMAHVTKRSIADVAASGPGLAFLAYPEAVTQLPISPLWAILFFSMLLMLGIDSQFCTVEGFITALVDEYPRLLRNRRELFIAAVCIISYLIGLSNITQGGIYVFKLFDYYSASGMSLLFLVFFECVSISWFYGVNRFYDNIQEMVGSRPCIWWKLCWSFFTPIIVAGVFIFSAVQMTPLTMGNYVFPKWGQGVGWLMALSSMVLIPGYMAYMFLTLKGSLKQRIQVMVQPSEDIVRPENGPEQPQAGSSTSKEAYI",
# ACHA5_HUMAN
"MAARGSGPRALRLLLLVQLVAGRCGLAGAAGGAQRGLSEPSSIAKHEDSLLKDLFQDYERWVRPVEHLNDKIKIKFGLAISQLVDVDEKNQLMTTNVWLKQEWIDVKLRWNPDDYGGIKVIRVPSDSVWTPDIVLFDNADGRFEGTSTKTVIRYNGTVTWTPPANYKSSCTIDVTFFPFDLQNCSMKFGSWTYDGSQVDIILEDQDVDKRDFFDNGEWEIVSATGSKGNRTDSCCWYPYVTYSFVIKRLPLFYTLFLIIPCIGLSFLTVLVFYLPSNEGEKICLCTSVLVSLTVFLLVIEEIIPSSSKVIPLIGEYLVFTMIFVTLSIMVTVFAINIHHRSSSTHNAMAPLVRKIFLHTLPKLLCMRSHVDRYFTQKEETESGSGPKSSRNTLEAALDSIRYITRHIMKENDVREVVEDWKFIAQVLDRMFLWTFLFVSIVGSLGLFVPVIYKWANILIPVHIGNANK",
# AA1R_HUMAN
"MPPSISAFQAAYIGIEVLIALVSVPGNVLVIWAVKVNQALRDATFCFIVSLAVADVAVGALVIPLAILINIGPQTYFHTCLMVACPVLILTQSSILALLAIAVDRYLRVKIPLRYKMVVTPRRAAVAIAGCWILSFVVGLTPMFGWNNLSAVERAWAANGSMGEPVIKCEFEKVISMEYMVYFNFFVWVLPPLLLMVLIYLEVFYLIRKQLNKKVSASSGDPQKYYGKELKIAKSLALILFLFALSWLPLHILNCITLFCPSCHKPSILTYIAIFLTHGNSAMNPIVYAFRIQKFRVTFLKIWNDHFRCQPAPPIDEDLPEERPDD",
# ACHB4_HUMAN
"MRRAPSLVLFFLVALCGRGNCRVANAEEKLMDDLLNKTRYNNLIRPATSSSQLISIKLQLSLAQLISVNEREQIMTTNVWLKQEWTDYRLTWNSSRYEGVNILRIPAKRIWLPDIVLYNNADGTYEVSVYTNLIVRSNGSVLWLPPAIYKSACKIEVKYFPFDQQNCTLKFRSWTYDHTEIDMVLMTPTASMDDFTPSGEWDIVALPGRRTVNPQDPSYVDVTYDFIIKRKPLFYTINLIIPCVLTTLLAILVFYLPSDCGEKMTLCISVLLALTFFLLLISKIVPPTSLDVPLIGKYLMFTMVLVTFSIVTSVCVLNVHHRSPSTHTMAPWVKRCFLHKLPTFLFMKRPGPDSSPARAFPPSKSCVTKPEATATSTSPSNFYGNSMYFVNPASAASKSPAGSTPVAIPRDFWLRSSGRFRQDVQEALEGVSFIAQHMKNDDEDQSVVEDWKYVAMVVDRLFLWVFMFVCVLGTVGLFLPPLFQTHAASEGPYAAQRD",
# COR1A_HUMAN
"MSRQVVRSSKFRHVFGQPAKADQCYEDVRVSQTTWDSGFCAVNPKFVALICEASGGGAFLVLPLGKTGRVDKNAPTVCGHTAPVLDIAWCPHNDNVIASGSEDCTVMVWEIPDGGLMLPLREPVVTLEGHTKRVGIVAWHTTAQNVLLSAGCDNVIMVWDVGTGAAMLTLGPEVHPDTIYSVDWSRDGGLICTSCRDKRVRIIEPRKGTVVAEKDRPHEGTRPVRAVFVSEGKILTTGFSRMSERQVALWDTKHLEEPLSLQELDTSSGVLLPFFDPDTNIVYLCGKGDSSIRYFEITSEAPFLHYLSMFSSKESQRGMGYMPKRGLEVNKCEIARFYKLHERRCEPIAMTVPRKSDLFQEDLYPPTAGPDPALTAEEWLGGRDAGPLLISLKDGYVPPKSRELRVNRGLDTGRRRAAPEASGTPSSDAVSRLEEEMRKLQATVQELQKRLDRLEETVQAK",
# GDIA_HUMAN
"MDEEYDVIVLGTGLTECILSGIMSVNGKKVLHMDRNPYYGGESSSITPLEELYKRFQLLEGPPESMGRGRDWNVDLIPKFLMANGQLVKMLLYTEVTRYLDFKVVEGSFVYKGGKIYKVPSTETEALASNLMGMFEKRRFRKFLVFVANFDENDPKTFEGVDPQTTSMRDVYRKFDLGQDVIDFTGHALALYRTDDYLDQPCLETVNRIKLYSESLARYGKSPYLYPLYGLGELPQGFARLSAIYGGTYMLNKPVDDIIMENGKVVGVKSEGEVARCKQLICDPSYIPDRVRKAGQVIRIICILSHPIKNTNDANSCQIIIPQNQVNRKSDIYVCMISYAHNVAAQGKYIAIASTTVETTDPEKEVEPALELLEPIDQKFVAISDLYEPIDDGCESQVFCSCSYDATTHFETTCNDIKDIYKRMAGTAFDFENMKRKQNDVFGEAEQ",
# KAP1_HUMAN
"MASPPACPSEEDESLKGCELYVQLHGIQQVLKDCIVHLCISKPERPMKFLREHFEKLEKEENRQILARQKSNSQSDSHDEEVSPTPPNPVVKARRRRGGVSAEVYTEEDAVSYVRKVIPKDYKTMTALAKAISKNVLFAHLDDNERSDIFDAMFPVTHIAGETVIQQGNEGDNFYVVDQGEVDVYVNGEWVTNISEGGSFGELALIYGTPRAATVKAKTDLKLWGIDRDSYRRILMGSTLRKRKMYEEFLSKVSILESLEKWERLTVADALEPVQFEDGEKIVVQGEPGDDFYIITEGTASVLQRRSPNEEYVEVGRLGPSDYFGEIALLLNRPRAATVVARGPLKCVKLDRPRFERVLGPCSEILKRNIQRYNSFISLTV",
# KAP3_HUMAN
"MSIEIPAGLTELLQGFTVEVLRHQPADLLEFALQHFTRLQQENERKGTARFGHEGRTWGDLGAAAGGGTPSKGVNFAEEPMQSDSEDGEEEEAAPADAGAFNAPVINRFTRRASVCAEAYNPDEEEDDAESRIIHPKTDDQRNRLQEACKDILLFKNLDPEQMSQVLDAMFEKLVKDGEHVIDQGDDGDNFYVIDRGTFDIYVKCDGVGRCVGNYDNRGSFGELALMYNTPRAATITATSPGALWGLDRVTFRRIIVKNNAKKRKMYESFIESLPFLKSLEFSERLKVVDVIGTKVYNDGEQIIAQGDSADSFFIVESGEVKITMKRKGKSEVEENGAVEIARCSRGQYFGELALVTNKPRAASAHAIGTVKCLAMDVQAFERLLGPCMEIMKRNIATYEEQLVALFGTNMDIVEPTA",
# SC6A6_HUMAN
"MATKEKLQCLKDFHKDILKPSPGKSPGTRPEDEAEGKPPQREKWSSKIDFVLSVAGGFVGLGNVWRFPYLCYKNGGGAFLIPYFIFLFGSGLPVFFLEIIIGQYTSEGGITCWEKICPLFSGIGYASVVIVSLLNVYYIVILAWATYYLFQSFQKELPWAHCNHSWNTPHCMEDTMRKNKSVWITISSTNFTSPVIEFWERNVLSLSPGIDHPGSLKWDLALCLLLVWLVCFFCIWKGVRSTGKVVYFTATFPFAMLLVLLVRGLTLPGAGAGIKFYLYPDITRLEDPQVWIDAGTQIFFSYAICLGAMTSLGSYNKYKYNSYRDCMLLGCLNSGTSFVSGFAIFSILGFMAQEQGVDIADVAESGPGLAFIAYPKAVTMMPLPTFWSILFFIMLLLLGLDSQFVEVEGQITSLVDLYPSFLRKGYRREIFIAFVCSISYLLGLTMVTEGGMYVFQLFDYYAASGVCLLWVAFFECFVIAWIYGGDNLYDGIEDMIGYRPGPWMKYSWAVITPVLCVGCFIFSLVKYVPLTYNKTYVYPNWAIGLGWSLALSSMLCVPLVIVIRLCQTEGPFLVRVKYLLTPREPNRWAVEREGATPYNSRTVMNGALVKPTHIIVETMM",
# GBRA5_HUMAN
"MDNGMFSGFIMIKNLLLFCISMNLSSHFGFSQMPTSSVKDETNDNITIFTRILDGLLDGYDNRLRPGLGERITQVRTDIYVTSFGPVSDTEMEYTIDVFFRQSWKDERLRFKGPMQRLPLNNLLASKIWTPDTFFHNGKKSIAHNMTTPNKLLRLEDDGTLLYTMRLTISAECPMQLEDFPMDAHACPLKFGSYAYPNSEVVYVWTNGSTKSVVVAEDGSRLNQYHLMGQTVGTENISTSTGEYTIMTAHFHLKRKIGYFVIQTYLPCIMTVILSQVSFWLNRESVPARTVFGVTTVLTMTTLSISARNSLPKVAYATAMDWFIAVCYAFVFSALIEFATVNYFTKRGWAWDGKKALEAAKIKKKREVILNKSTNAFTTGKMSHPPNIPKEQTPAGTSNTTSVSVKPSEEKTSESKKTYNSISKIDKMSRIVFPVLFGTFNLVYWATYLNREPVIKGAASPK",
# SC6A4_HUMAN
"METTPLNSQKQLSACEDGEDCQENGVLQKVVPTPGDKVESGQISNGYSAVPSPGAGDDTRHSIPATTTTLVAELHQGERETWGKKVDFLLSVIGYAVDLGNVWRFPYICYQNGGGAFLLPYTIMAIFGGIPLFYMELALGQYHRNGCISIWRKICPIFKGIGYAICIIAFYIASYYNTIMAWALYYLISSFTDQLPWTSCKNSWNTGNCTNYFSEDNITWTLHSTSPAEEFYTRHVLQIHRSKGLQDLGGISWQLALCIMLIFTVIYFSIWKGVKTSGKVVWVTATFPYIILSVLLVRGATLPGAWRGVLFYLKPNWQKLLETGVWIDAAAQIFFSLGPGFGVLLAFASYNKFNNNCYQDALVTSVVNCMTSFVSGFVIFTVLGYMAEMRNEDVSEVAKDAGPSLLFITYAEAIANMPASTFFAIIFFLMLITLGLDSTFAGLEGVITAVLDEFPHVWAKRRERFVLAVVITCFFGSLVTLTFGGAYVVKLLEEYATGPAVLTVALIEAVAVSWFYGITQFCRDVKEMLGFSPGWFWRICWVAISPLFLLFIICSFLMSPPQLRLFQYNYPYWSIILGYCIGTSSFICIPTYIAYRLIITPGTFKERIIKSITPETPTEIPCGDIRLNAV",
# AKT1_HUMAN
"MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVDQREAPLNNFSVAQCQLMKTERPRPNTFIIRCLQWTTVIERTFHVETPEEREEWTTAIQTVADGLKKQEEEEMDFRSGSPSDNSGAEEMEVSLAKPKHRVTMNEFEYLKLLGKGTFGKVILVKEKATGRYYAMKILKKEVIVAKDEVAHTLTENRVLQNSRHPFLTALKYSFQTHDRLCFVMEYANGGELFFHLSRERVFSEDRARFYGAEIVSALDYLHSEKNVVYRDLKLENLMLDKDGHIKITDFGLCKEGIKDGATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHEKLFELILMEEIRFPRTLGPEAKSLLSGLLKKDPKQRLGGGSEDAKEIMQHRFFAGIVWQHVYEKKLSPPFKPQVTSETDTRYFDEEFTAQMITITPPDQDDSMECVDSERRPHFPQFSYSASGTA",
# HNRH1_HUMAN
"MMLGTEGGEGFVVKVRGLPWSCSADEVQRFFSDCKIQNGAQGIRFIYTREGRPSGEAFVELESEDEVKLALKKDRETMGHRYVEVFKSNNVEMDWVLKHTGPNSPDTANDGFVRLRGLPFGCSKEEIVQFFSGLEIVPNGITLPVDFQGRSTGEAFVQFASQEIAEKALKKHKERIGHRYIEIFKSSRAEVRTHYDPPRKLMAMQRPGPYDRPGAGRGYNSIGRGAGFERMRRGAYGGGYGGYDDYNGYNDGYGFGSDRFGRDLNYCFSGMSDHRYGDGGSTFQSTTGHCVHMRGLPYRATENDIYNFFSPLNPVRVHIEIGPDGRVTGEADVEFATHEDAVAAMSKDKANMQHRYVELFLNSTAGASGGAYEHRYVELFLNSTAGASGGAYGSQMMGGMGLSNQSSYGGPASQQLSGGYGGGYGGQSSMSGYDQVLQENSSDFQSNIA",
# L1CAM_HUMAN
"MVVALRYVWPLLLCSPCLLIQIPEEYEGHHVMEPPVITEQSPRRLVVFPTDDISLKCEASGKPEVQFRWTRDGVHFKPKEELGVTVYQSPHSGSFTITGNNSNFAQRFQGIYRCFASNKLGTAMSHEIRLMAEGAPKWPKETVKPVEVEEGESVVLPCNPPPSAEPLRIYWMNSKILHIKQDERVTMGQNGNLYFANVLTSDNHSDYICHAHFPGTRTIIQKEPIDLRVKATNSMIDRKPRLLFPTNSSSHLVALQGQPLVLECIAEGFPTPTIKWLRPSGPMPADRVTYQNHNKTLQLLKVGEEDDGEYRCLAENSLGSARHAYYVTVEAAPYWLHKPQSHLYGPGETARLDCQVQGRPQPEVTWRINGIPVEELAKDQKYRIQRGALILSNVQPSDTMVTQCEARNRHGLLLANAYIYVVQLPAKILTADNQTYMAVQGSTAYLLCKAFGAPVPSVQWLDEDGTTVLQDERFFPYANGTLGIRDLQANDTGRYFCLAANDQNNVTIMANLKVKDATQITQGPRSTIEKKGSRVTFTCQASFDPSLQPSITWRGDGRDLQELGDSDKYFIEDGRLVIHSLDYSDQGNYSCVASTELDVVESRAQLLVVGSPGPVPRLVLSDLHLLTQSQVRVSWSPAEDHNAPIEKYDIEFEDKEMAPEKWYSLGKVPGNQTSTTLKLSPYVHYTFRVTAINKYGPGEPSPVSETVVTPEAAPEKNPVDVKGEGNETTNMVITWKPLRWMDWNAPQVQYRVQWRPQGTRGPWQEQIVSDPFLVVSNTSTFVPYEIKVQAVNSQGKGPEPQVTIGYSGEDYPQAIPELEGIEILNSSAVLVKWRPVDLAQVKGHLRGYNVTYWREGSQRKHSKRHIHKDHVVVPANTTSVILSGLRPYSSYHLEVQAFNGRGSGPASEFTFSTPEGVPGHPEALHLECQSNTSLLLRWQPPLSHNGVLTGYVLSYHPLDEGGKGQLSFNLRDPELRTHNLTDLSPHLRYRFQLQATTKEGPGEAIVREGGTMALSGISDFGNISATAGENYSVVSWVPKEGQCNFRFHILFKALGEEKGGASLSPQYVSYNQSSYTQWDLQPDTDYEIHLFKERMFRHQMAVKTNGTGRVRLPPAGFATEGWFIGFVSAIILLLLVLLILCFIKRSKGGKYSVKDKEDTQVDSEARPMKDETFGEYRSLESDNEEKAFGSSQPSLNGDIKPLGSDDSLADYGGSVDVQFNEDGSFIGQYSGKKEKEAAGGNDSSGATSPINPAVALE",
# ARRB2_HUMAN
"MGEKPGTRVFKKSSPNCKLTVYLGKRDFVDHLDKVDPVDGVVLVDPDYLKDRKVFVTLTCAFRYGREDLDVLGLSFRKDLFIATYQAFPPVPNPPRPPTRLQDRLLRKLGQHAHPFFFTIPQNLPCSVTLQPGPEDTGKACGVDFEIRAFCAKSLEEKSHKRNSVRLVIRKVQFAPEKPGPQPSAETTRHFLMSDRSLHLEASLDKELYYHGEPLNVNVHVTNNSTKTVKKIKVSVRQYADICLFSTAQYKCPVAQLEQDDQVSPSSTFCKVYTITPLLSDNREKRGLALDGKLKHEDTNLASSTIVKEGANKEVLGILVSYRVKVKLVVSRGGDVSVELPFVLMHPKPHDHIPLPRPQSAAPETDVPVDTNLIEFDTNYATDDDIVFEDFARLRLKGMKDDDYDDQLC",
# ACHA3_HUMAN
"MGSGPLSLPLALSPPRLLLLLLLSLLPVARASEAEHRLFERLFEDYNEIIRPVANVSDPVIIHFEVSMSQLVKVDEVNQIMETNLWLKQIWNDYKLKWNPSDYGGAEFMRVPAQKIWKPDIVLYNNAVGDFQVDDKTKALLKYTGEVTWIPPAIFKSSCKIDVTYFPFDYQNCTMKFGSWSYDKAKIDLVLIGSSMNLKDYWESGEWAIIKAPGYKHDIKYNCCEEIYPDITYSLYIRRLPLFYTINLIIPCLLISFLTVLVFYLPSDCGEKVTLCISVLLSLTVFLLVITETIPSTSLVIPLIGEYLLFTMIFVTLSIVITVFVLNVHYRTPTTHTMPSWVKTVFLNLLPRVMFMTRPTSNEGNAQKPRPLYGAELSNLNCFSRAESKGCKEGYPCQDGMCGYCHHRRIKISNFSANLTRSSSSESVDAVLSLSALSPEIKEAIQSVKYIAENMKAQNEAKEIQDDWKYVAMVIDRIFLWVFTLVCILGTAGLFLQPLMAREDA",
# NAC1_HUMAN
"MYNMRRLSLSPTFSMGFHLLVTVSLLFSHVDHVIAETEMEGEGNETGECTGSYYCKKGVILPIWEPQDPSFGDKIARATVYFVAMVYMFLGVSIIADRFMSSIEVITSQEKEITIKKPNGETTKTTVRIWNETVSNLTLMALGSSAPEILLSVIEVCGHNFTAGDLGPSTIVGSAAFNMFIIIALCVYVVPDGETRKIKHLRVFFVTAAWSIFAYTWLYIILSVISPGVVEVWEGLLTFFFFPICVVFAWVADRRLLFYKYVYKRYRAGKQRGMIIEHEGDRPSSKTEIEMDGKVVNSHVENFLDGALVLEVDERDQDDEEARREMARILKELKQKHPDKEIEQLIELANYQVLSQQQKSRAFYRIQATRLMTGAGNILKRHAADQARKAVSMHEVNTEVTENDPVSKIFFEQGTYQCLENCGTVALTIIRRGGDLTNTVFVDFRTEDGTANAGSDYEFTEGTVVFKPGDTQKEIRVGIIDDDIFEEDENFLVHLSNVKVSSEASEDGILEANHVSTLACLGSPSTATVTIFDDDHAGIFTFEEPVTHVSESIGIMEVKVLRTSGARGNVIVPYKTIEGTARGGGEDFEDTCGELEFQNDEIVKTISVKVIDDEEYEKNKTFFLEIGEPRLVEMSEKKALLLNELGGFTITGKYLFGQPVFRKVHAREHPILSTVITIADEYDDKQPLTSKEEEERRIAEMGRPILGEHTKLEVIIEESYEFKSTVDKLIKKTNLALVVGTNSWREQFIEAITVSAGEDDDDDECGEEKLPSCFDYVMHFLTVFWKVLFAFVPPTEYWNGWACFIVSILMIGLLTAFIGDLASHFGCTIGLKDSVTAVVFVALGTSVPDTFASKVAATQDQYADASIGNVTGSNAVNVFLGIGVAWSIAAIYHAANGEQFKVSPGTLAFSVTLFTIFAFINVGVLLYRRRPEIGGELGGPRTAKLLTSCLFVLLWLLYIFFSSLEAYCHIKGF",
# STX2_HUMAN
"MRDRLPDLTACRKNDDGDTVVVVEKDHFMDDFFHQVEEIRNSIDKITQYVEEVKKNHSIILSAPNPEGKIKEELEDLNKEIKKTANKIRAKLKAIEQSFDQDESGNRTSVDLRIRRTQHSVLSRKFVEAMAEYNEAQTLFRERSKGRIQRQLEITGRTTTDDELEEMLESGKPSIFTSDIISDSQITRQALNEIESRHKDIMKLETSIRELHEMFMDMAMFVETQGEMINNIERNVMNATDYVEHAKEETKKAIKYQSKARRKKWIIIAVSVVLVAIIALIIGLSVGK",
# RL9_HUMAN
"MKTILSNQTVDIPENVDITLKGRTVIVKGPRGTLRRDFNHINVELSLLGKKKKRLRVDKWWGNRKELATVRTICSHVQNMIKGVTLGFRYKMRSVYAHFPINVVIQENGSLVEIRNFLGEKYIRRVRMRPGVACSVSQAQKDELILEGNDIELVSNSAALIQQATTVKNKDIRKFLDGIYVSEKGTVQQADE",
# KINH_HUMAN
"MADLAECNIKVMCRFRPLNESEVNRGDKYIAKFQGEDTVVIASKPYAFDRVFQSSTSQEQVYNDCAKKIVKDVLEGYNGTIFAYGQTSSGKTHTMEGKLHDPEGMGIIPRIVQDIFNYIYSMDENLEFHIKVSYFEIYLDKIRDLLDVSKTNLSVHEDKNRVPYVKGCTERFVCSPDEVMDTIDEGKSNRHVAVTNMNEHSSRSHSIFLINVKQENTQTEQKLSGKLYLVDLAGSEKVSKTGAEGAVLDEAKNINKSLSALGNVISALAEGSTYVPYRDSKMTRILQDSLGGNCRTTIVICCSPSSYNESETKSTLLFGQRAKTIKNTVCVNVELTAEQWKKKYEKEKEKNKILRNTIQWLENELNRWRNGETVPIDEQFDKEKANLEAFTVDKDITLTNDKPATAIGVIGNFTDAERRKCEEEIAKLYKQLDDKDEEINQQSQLVEKLKTQMLDQEELLASTRRDQDNMQAELNRLQAENDASKEEVKEVLQALEELAVNYDQKSQEVEDKTKEYELLSDELNQKSATLASIDAELQKLKEMTNHQKKRAAEMMASLLKDLAEIGIAVGNNDVKQPEGTGMIDEEFTVARLYISKMKSEVKTMVKRCKQLESTQTESNKKMEENEKELAACQLRISQHEAKIKSLTEYLQNVEQKKRQLEESVDALSEELVQLRAQEKVHEMEKEHLNKVQTANEVKQAVEQQIQSHRETHQKQISSLRDEVEAKAKLITDLQDQNQKMMLEQERLRVEHEKLKATDQEKSRKLHELTVMQDRREQARQDLKGLEETVAKELQTLHNLRKLFVQDLATRVKKSAEIDSDDTGGSAAQKQKISFLENNLEQLTKVHKQLVRDNADLRCELPKLEKRLRATAERVKALESALKEAKENASRDRKRYQQEVDRIKEAVRSKNMARRGHSAQIAKPIRPGQHPAASPTHPSAIRGGGAFVQNSQPVAVRGGGGKQV",
# SDC2_HUMAN
"MRRAWILLTLGLVACVSAESRAELTSDKDMYLDNSSIEEASGVYPIDDDDYASASGSGADEDVESPELTTSRPLPKILLTSAAPKVETTTLNIQNKIPAQTKSPEETDKEKVHLSDSERKMDPAEEDTNVYTEKHSDSLFKRTEVLAAVIAGGVIGFLFAIFLILLLVYRMRKKDEGSYDLGERKPSSAAYQKAPTKEFYA",
# GBRA3_HUMAN
"MIITQTSHCYMTSLGILFLINILPGTTGQGESRRQEPGDFVKQDIGGLSPKHAPDIPDDSTDNITIFTRILDRLLDGYDNRLRPGLGDAVTEVKTDIYVTSFGPVSDTDMEYTIDVFFRQTWHDERLKFDGPMKILPLNNLLASKIWTPDTFFHNGKKSVAHNMTTPNKLLRLVDNGTLLYTMRLTIHAECPMHLEDFPMDVHACPLKFGSYAYTTAEVVYSWTLGKNKSVEVAQDGSRLNQYDLLGHVVGTEIIRSSTGEYVVMTTHFHLKRKIGYFVIQTYLPCIMTVILSQVSFWLNRESVPARTVFGVTTVLTMTTLSISARNSLPKVAYATAMDWFIAVCYAFVFSALIEFATVNYFTKRSWAWEGKKVPEALEMKKKTPAAPAKKTSTTFNIVGTTYPINLAKDTEFSTISKGAAPSASSTPTIIASPKATYVQDSPTETKTYNSVSKVDKISRIIFPVLFAIFNLVYWATYVNRESAIKGMIRKQ",
# RYK_HUMAN
"MRGAARLGRPGRSCLPGARGLRAPPPPPLLLLLALLPLLPAPGAAAAPAPRPPELQSASAGPSVSLYLSEDEVRRLIGLDAELYYVRNDLISHYALSFSLLVPSETNFLHFTWHAKSKVEYKLGFQVDNVLAMDMPQVNISVQGEVPRTLSVFRVELSCTGKVDSEVMILMQLNLTVNSSKNFTVLNFKRRKMCYKKLEEVKTSALDKNTSRTIYDPVHAAPTTSTRVFYISVGVCCAVIFLVAIILAVLHLHSMKRIELDDSISASSSSQGLSQPSTQTTQYLRADTPNNATPITSYPTLRIEKNDLRSVTLLEAKGKVKDIAISRERITLKDVLQEGTFGRIFHGILIDEKDPNKEKQAFVKTVKDQASEIQVTMMLTESCKLRGLHHRNLLPITHVCIEEGEKPMVILPYMNWGNLKLFLRQCKLVEANNPQAISQQDLVHMAIQIACGMSYLARREVIHKDLAARNCVIDDTLQVKITDNALSRDLFPMDYHCLGDNENRPVRWMALESLVNNEFSSASDVWAFGVTLWELMTLGQTPYVDIDPFEMAAYLKDGYRIAQPINCPDELFAVMACCWALDPEERPKFQQLVQCLTEFHAALGAYV",
# 5HT7R_HUMAN
"MMDVNSSGRPDLYGHLRSFLLPEVGRGLPDLSPDGGADPVAGSWAPHLLSEVTASPAPTWDAPPDNASGCGEQINYGRVEKVVIGSILTLITLLTIAGNCLVVISVCFVKKLRQPSNYLIVSLALADLSVAVAVMPFVSVTDLIGGKWIFGHFFCNVFIAMDVMCCTASIMTLCVISIDRYLGITRPLTYPVRQNGKCMAKMILSVWLLSASITLPPLFGWAQNVNDDKVCLISQDFGYTIYSTAVAFYIPMSVMLFMYYQIYKAARKSAAKHKFPGFPRVEPDSVIALNGIVKLQKEVEECANLSRLLKHERKNISIFKREQKAATTLGIIVGAFTVCWLPFFLLSTARPFICGTSCSCIPLWVERTFLWLGYANSLINPFIYAFFNRDLRTTYRSLLQCQYRNINRKLSAAGMHEALKLAERPERPEFVLRACTRRVLLRPEKRPPVSVWVLQSPDHHNWLADKMLTTVEKKVMIHD",
# CNR2_HUMAN
"MEECWVTEIANGSKDGLDSNPMKDYMILSGPQKTAVAVLCTLLGLLSALENVAVLYLILSSHQLRRKPSYLFIGSLAGADFLASVVFACSFVNFHVFHGVDSKAVFLLKIGSVTMTFTASVGSLLLTAIDRYLCLRYPPSYKALLTRGRALVTLGIMWVLSALVSYLPLMGWTCCPRPCSELFPLIPNDYLLSWLLFIAFLFSGIIYTYGHVLWKAHQHVASLSGHQDRQVPGMARMRLDVRLAKTLGLVLAVLLICWFPVLALMAHSLATTLSDQVKKAFAFCSMLCLINSMVNPVIYALRSGEIRSSAHHCLAHWKKCVRGLGSEAKEEAPRSSVTETEADGKITPWPDSRDLDLSDC",
# CRFR1_HUMAN
"MGGHPQLRLVKALLLLGLNPVSASLQDQHCESLSLASNISGLQCNASVDLIGTCWPRSPAGQLVVRPCPAFFYGVRYNTTNNGYRECLANGSWAARVNYSECQEILNEEKKSKVHYHVAVIINYLGHCISLVALLVAFVLFLRLRSIRCLRNIIHWNLISAFILRNATWFVVQLTMSPEVHQSNVGWCRLVTAAYNYFHVTNFFWMFGEGCYLHTAIVLTYSTDRLRKWMFICIGWGVPFPIIVAWAIGKLYYDNEKCWFGKRPGVYTDYIYQGPMILVLLINFIFLFNIVRILMTKLRASTTSETIQYRKAVKATLVLLPLLGITYMLFFVNPGEDEVSRVVFIYFNSFLESFQGFFVSVFYCFLNSEVRSAIRKRWHRWQDKHSIRARVARAMSIPTSPTRVSFHSIKQSTAV",
# GPC1_HUMAN
"MELRARGWWLLCAAAALVACARGDPASKSRSCGEVRQIYGAKGFSLSDVPQAEISGEHLRICPQGYTCCTSEMEENLANRSHAELETALRDSSRVLQAMLATQLRSFDDHFQHLLNDSERTLQATFPGAFGELYTQNARAFRDLYSELRLYYRGANLHLEETLAEFWARLLERLFKQLHPQLLLPDDYLDCLGKQAEALRPFGEAPRELRLRATRAFVAARSFVQGLGVASDVVRKVAQVPLGPECSRAVMKLVYCAHCLGVPGARPCPDYCRNVLKGCLANQADLDAEWRNLLDSMVLITDKFWGTSGVESVIGSVHTWLAEAINALQDNRDTLTAKVIQGCGNPKVNPQGPGPEEKRRRGKLAPRERPPSGTLEKLVSEAKAQLRDVQDFWISLPGTLCSEKMALSTASDDRCWNGMARGRYLPEVMGDGLANQINNPEVEVDITKPDMTIRQQIMQLKIMTNRLRSAYNGNDVDFQDASDDGSGSGSGDGCLDDLCSRKVSRKSSSSRTPLTHALPGLSEQEGQKTSAASCPQPPTFLLPLLLFLALTVARPRWR",
# PROF2_HUMAN
"MAGWQSYVDNLMCDGCCQEAAIVGYCDAKYVWAATAGGVFQSITPIEIDMIVGKDREGFFTNGLTLGAKKCSVIRDSLYVDGDCTMDIRTKSQGGEPTYNVAVGRAGRVLVFVMGKEGVHGGGLNKKAYSMAKYLRDSGF",
# UBP6_HUMAN
"MDMVENADSLQAQERKDILMKYDKGHRAGLPEDKGPEPVGINSSIDRFGILHETELPPVTAREAKKIRREMTRTSKWMEMLGEWETYKHSSKLIDRVYKGIPMNIRGPVWSVLLNIQEIKLKNPGRYQIMKERGKRSSEHIHHIDLDVRTTLRNHVFFRDRYGAKQRELFYILLAYSEYNPEVGYCRDLSHITALFLLYLPEEDAFWALVQLLASERHSLPGFHSPNGGTVQGLQDQQEHVVPKSQPKTMWHQDKEGLCGQCASLGCLLRNLIDGISLGLTLRLWDVYLVEGEQVLMPITSIALKVQQKRLMKTSRCGLWARLRNQFFDTWAMNDDTVLKHLRASTKKLTRKQGDLPPPAKREQGSLAPRPVPASRGGKTLCKGYRQAPPGPPAQFQRPICSASPPWASRFSTPCPGGAVREDTYPVGTQGVPSLALAQGGPQGSWRFLEWKSMPRLPTDLDIGGPWFPHYDFEWSCWVRAISQEDQLATCWQAEHCGEVHNKDMSWPEEMSFTANSSKIDRQKVPTEKGATGLSNLGNTCFMNSSIQCVSNTQPLTQYFISGRHLYELNRTNPIGMKGHMAKCYGDLVQELWSGTQKSVAPLKLRRTIAKYAPKFDGFQQQDSQELLAFLLDGLHEDLNRVHEKPYVELKDSDGRPDWEVAAEAWDNHLRRNRSIIVDLFHGQLRSQVKCKTCGHISVRFDPFNFLSLPLPMDSYMDLEITVIKLDGTTPVRYGLRLNMDEKYTGLKKQLRDLCGLNSEQILLAEVHDSNIKNFPQDNQKVQLSVSGFLCAFEIPVPSSPISASSPTQIDFSSSPSTNGMFTLTTNGDLPKPIFIPNGMPNTVVPCGTEKNFTNGMVNGHMPSLPDSPFTGYIIAVHRKMMRTELYFLSPQENRPSLFGMPLIVPCTVHTRKKDLYDAVWIQVSWLARPLPPQEASIHAQDRDNCMGYQYPFTLRVVQKDGNSCAWCPQYRFCRGCKIDCGEDRAFIGNAYIAVDWHPTALHLRYQTSQERVVDKHESVEQSRRAQAEPINLDSCLRAFTSEEELGESEMYYCSKCKTHCLATKKLDLWRLPPFLIIHLKRFQFVNDQWIKSQKIVRFLRESFDPSAFLVPRDPALCQHKPLTPQGDELSKPRILAREVKKVDAQSSAGKEDMLLSKSPSSLSANISSSPKGSPSSSRKSGTSCPSSKNSSPNSSPRTLGRSKGRLRLPQIGSKNKPSSSKKNLDASKENGAGQICELADALSRGHMRGGSQPELVTPQDHEVALANGFLYEHEACGNGCGDGYSNGQLGNHSEEDSTDDQREDTHIKPIYNLYAISCHSGILSGGHYITYAKNPNCKWYCYNDSSCEELHPDEIDTDSAYILFYEQQGIDYAQFLPKIDGKKMADTSSTDEDSESDYEKYSMLQ",
# CTNB1_HUMAN
"MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSLSGKGNPEEEDVDTSQVLYEWEQGFSQSFTQEQVADIDGQYAMTRAQRVRAAMFPETLDEGMQIPSTQFDAAHPTNVQRLAEPSQMLKHAVVNLINYQDDAELATRAIPELTKLLNDEDQVVVNKAAVMVHQLSKKEASRHAIMRSPQMVSAIVRTMQNTNDVETARCTAGTLHNLSHHREGLLAIFKSGGIPALVKMLGSPVDSVLFYAITTLHNLLLHQEGAKMAVRLAGGLQKMVALLNKTNVKFLAITTDCLQILAYGNQESKLIILASGGPQALVNIMRTYTYEKLLWTTSRVLKVLSVCSSNKPAIVEAGGMQALGLHLTDPSQRLVQNCLWTLRNLSDAATKQEGMEGLLGTLVQLLGSDDINVVTCAAGILSNLTCNNYKNKMMVCQVGGIEALVRTVLRAGDREDITEPAICALRHLTSRHQEAEMAQNAVRLHYGLPVVVKLLHPPSHWPLIKATVGLIRNLALCPANHAPLREQGAIPRLVQLLVRAHQDTQRRTSMGGTQQQFVEGVRMEEIVEGCTGALHILARDVHNRIVIRGLNTIPLFVQLLYSPIENIQRVAAGVLCELAQDKEAAEAIEAEGATAPLTELLHSRNEGVATYAAAVLFRMSEDKPQDYKKRLSVELTSSLFRTEPMAWNETADLGLDIGAQGEPLGYRQDDPSYRSFHSGGYGQDALGMDPMMEHEMGGHHPGADYPVDGLPDLGHAQDLMDGLPPGDSNQLAWFDTDL",
# PHB1_HUMAN
"MAAKVFESIGKFGLALAVAGGVVNSALYNVDAGHRAVIFDRFRGVQDIVVGEGTHFLIPWVQKPIIFDCRSRPRNVPVITGSKDLQNVNITLRILFRPVASQLPRIFTSIGEDYDERVLPSITTEILKSVVARFDAGELITQRELVSRQVSDDLTERAATFGLILDDVSLTHLTFGKEFTEAVEAKQVAQQEAERARFVVEKAEQQKKAAIISAEGDSKAAELIANSLATAGDGLIELRKLEAAEDIAYQLSRSRNITYLPAGQSVLLQLPQ",
# RADI_HUMAN
"MPKPINVRVTTMDAELEFAIQPNTTGKQLFDQVVKTVGLREVWFFGLQYVDSKGYSTWLKLNKKVTQQDVKKENPLQFKFRAKFFPEDVSEELIQEITQRLFFLQVKEAILNDEIYCPPETAVLLASYAVQAKYGDYNKEIHKPGYLANDRLLPQRVLEQHKLTKEQWEERIQNWHEEHRGMLREDSMMEYLKIAQDLEMYGVNYFEIKNKKGTELWLGVDALGLNIYEHDDKLTPKIGFPWSEIRNISFNDKKFVIKPIDKKAPDFVFYAPRLRINKRILALCMGNHELYMRRRKPDTIEVQQMKAQAREEKHQKQLERAQLENEKKKREIAEKEKERIEREKEELMERLKQIEEQTIKAQKELEEQTRKALELDQERKRAKEEAERLEKERRAAEEAKSAIAKQAADQMKNQEQLAAELAEFTAKIALLEEAKKKKEEEATEWQHKAFAAQEDLEKTKEELKTVMSAPPPPPPPPVIPPTENEHDEHDENNAEASAELSNEGVMNHRSEEERVTETQKNERVKKQLQALSSELAQARDETKKTQNDVLHAENVKAGRDKYKTLRQIRQGNTKQRIDEFEAM",
# RL22_HUMAN
"MAPVKKLVVKGGKKKKQVLKFTLDCTHPVEDGIMDAANFEQFLQERIKVNGKAGNLGGGVVTIERSKSKITVTSEVPFSKRYLKYLTKKYLKKNNLRDWLRVVANSKESYELRYFQINQDEEEEEDED",
# ADA1A_HUMAN
"MVFLSGNASDSSNCTQPPAPVNISKAILLGVILGGLILFGVLGNILVILSVACHRHLHSVTHYYIVNLAVADLLLTSTVLPFSAIFEVLGYWAFGRVFCNIWAAVDVLCCTASIMGLCIISIDRYIGVSYPLRYPTIVTQRRGLMALLCVWALSLVISIGPLFGWRQPAPEDETICQINEEPGYVLFSALGSFYLPLAIILVMYCRVYVVAKRESRGLKSGLKTDKSDSEQVTLRIHRKNAPAGGSGMASAKTKTHFSVRLLKFSREKKAAKTLGIVVGCFVLCWLPFFLVMPIGSFFPDFKPSETVFKIVFWLGYLNSCINPIIYPCSSQEFKKAFQNVLRIQCLCRKQSSKHALGYTLHPPSQAVEGQHKDMVRIPVGSRETFYRISKTDGVCEWKFFSSMPRGSARITVSKDQSSCTTARVRSKSFLQVCCCVGPSTPSLDKNHQVPTIKVHTISLSENGEEV",
# OPRM_HUMAN
"MDSSAAPTNASNCTDALAYSSCSPAPSPGSWVNLSHLDGNLSDPCGPNRTDLGGRDSLCPPTGSPSMITAITIMALYSIVCVVGLFGNFLVMYVIVRYTKMKTATNIYIFNLALADALATSTLPFQSVNYLMGTWPFGTILCKIVISIDYYNMFTSIFTLCTMSVDRYIAVCHPVKALDFRTPRNAKIINVCNWILSSAIGLPVMFMATTKYRQGSIDCTLTFSHPTWYWENLLKICVFIFAFIMPVLIITVCYGLMILRLKSVRMLSGSKEKDRNLRRITRMVLVVVAVFIVCWTPIHIYVIIKALVTIPETTFQTVSWHFCIALGYTNSCLNPVLYAFLDENFKRCFREFCIPTSSNIEQQNSTRIRQNTRDHPSTANTVDRTNHQLENLEAETAPLP",
# DRD3_HUMAN
"MASLSQLSGHLNYTCGAENSTGASQARPHAYYALSYCALILAIVFGNGLVCMAVLKERALQTTTNYLVVSLAVADLLVATLVMPWVVYLEVTGGVWNFSRICCDVFVTLDVMMCTASILNLCAISIDRYTAVVMPVHYQHGTGQSSCRRVALMITAVWVLAFAVSCPLLFGFNTTGDPTVCSISNPDFVIYSSVVSFYLPFGVTVLVYARIYVVLKQRRRKRILTRQNSQCNSVRPGFPQQTLSPDPAHLELKRYYSICQDTALGGPGFQERGGELKREEKTRNSLSPTIAPKLSLEVRKLSNGRLSTSLKLGPLQPRGVPLREKKATQMVAIVLGAFIVCWLPFFLTHVLNTHCQTCHVSPELYSATTWLGYVNSALNPVIYTTFNIEFRKAFLKILSC",
# SCN1A_HUMAN
"MEQTVLVPPGPDSFNFFTRESLAAIERRIAEEKAKNPKPDKKDDDENGPKPNSDLEAGKNLPFIYGDIPPEMVSEPLEDLDPYYINKKTFIVLNKGKAIFRFSATSALYILTPFNPLRKIAIKILVHSLFSMLIMCTILTNCVFMTMSNPPDWTKNVEYTFTGIYTFESLIKIIARGFCLEDFTFLRDPWNWLDFTVITFAYVTEFVDLGNVSALRTFRVLRALKTISVIPGLKTIVGALIQSVKKLSDVMILTVFCLSVFALIGLQLFMGNLRNKCIQWPPTNASLEEHSIEKNITVNYNGTLINETVFEFDWKSYIQDSRYHYFLEGFLDALLCGNSSDAGQCPEGYMCVKAGRNPNYGYTSFDTFSWAFLSLFRLMTQDFWENLYQLTLRAAGKTYMIFFVLVIFLGSFYLINLILAVVAMAYEEQNQATLEEAEQKEAEFQQMIEQLKKQQEAAQQAATATASEHSREPSAAGRLSDSSSEASKLSSKSAKERRNRRKKRKQKEQSGGEEKDEDEFQKSESEDSIRRKGFRFSIEGNRLTYEKRYSSPHQSLLSIRGSLFSPRRNSRTSLFSFRGRAKDVGSENDFADDEHSTFEDNESRRDSLFVPRRHGERRNSNLSQTSRSSRMLAVFPANGKMHSTVDCNGVVSLVGGPSVPTSPVGQLLPEVIIDKPATDDNGTTTETEMRKRRSSSFHVSMDFLEDPSQRQRAMSIASILTNTVEELEESRQKCPPCWYKFSNIFLIWDCSPYWLKVKHVVNLVVMDPFVDLAITICIVLNTLFMAMEHYPMTDHFNNVLTVGNLVFTGIFTAEMFLKIIAMDPYYYFQEGWNIFDGFIVTLSLVELGLANVEGLSVLRSFRLLRVFKLAKSWPTLNMLIKIIGNSVGALGNLTLVLAIIVFIFAVVGMQLFGKSYKDCVCKIASDCQLPRWHMNDFFHSFLIVFRVLCGEWIETMWDCMEVAGQAMCLTVFMMVMVIGNLVVLNLFLALLLSSFSADNLAATDDDNEMNNLQIAVDRMHKGVAYVKRKIYEFIQQSFIRKQKILDEIKPLDDLNNKKDSCMSNHTAEIGKDLDYLKDVNGTTSGIGTGSSVEKYIIDESDYMSFINNPSLTVTVPIAVGESDFENLNTEDFSSESDLEESKEKLNESSSSSEGSTVDIGAPVEEQPVVEPEETLEPEACFTEGCVQRFKCCQINVEEGRGKQWWNLRRTCFRIVEHNWFETFIVFMILLSSGALAFEDIYIDQRKTIKTMLEYADKVFTYIFILEMLLKWVAYGYQTYFTNAWCWLDFLIVDVSLVSLTANALGYSELGAIKSLRTLRALRPLRALSRFEGMRVVVNALLGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFYHCINTTTGDRFDIEDVNNHTDCLKLIERNETARWKNVKVNFDNVGFGYLSLLQVATFKGWMDIMYAAVDSRNVELQPKYEESLYMYLYFVIFIIFGSFFTLNLFIGVIIDNFNQQKKKFGGQDIFMTEEQKKYYNAMKKLGSKKPQKPIPRPGNKFQGMVFDFVTRQVFDISIMILICLNMVTMMVETDDQSEYVTTILSRINLVFIVLFTGECVLKLISLRHYYFTIGWNIFDFVVVILSIVGMFLAELIEKYFVSPTLFRVIRLARIGRILRLIKGAKGIRTLLFALMMSLPALFNIGLLLFLVMFIYAIFGMSNFAYVKREVGIDDMFNFETFGNSMICLFQITTSAGWDGLLAPILNSKPPDCDPNKVNPGSSVKGDCGNPSVGIFFFVSYIIISFLVVVNMYIAVILENFSVATEESAEPLSEDDFEMFYEVWEKFDPDATQFMEFEKLSQFAAALEPPLNLPQPNKLQLIAMDLPMVSGDRIHCLDILFAFTKRVLGESGEMDALRIQMEERFMASNPSKVSYQPITTTLKRKQEEVSAVIIQRAYRRHLLKRTVKQASFTYNKNKIKGGANLLIKEDMIIDRINENSITEKTDLTMSTAACPPSYDRVTKPIVEKHEQEGKDEKAKGK",
# MYH10_HUMAN
"MAQRTGLEDPERYLFVDRAVIYNPATQADWTAKKLVWIPSERHGFEAASIKEERGDEVMVELAENGKKAMVNKDDIQKMNPPKFSKVEDMAELTCLNEASVLHNLKDRYYSGLIYTYSGLFCVVINPYKNLPIYSENIIEMYRGKKRHEMPPHIYAISESAYRCMLQDREDQSILCTGESGAGKTENTKKVIQYLAHVASSHKGRKDHNIPGELERQLLQANPILESFGNAKTVKNDNSSRFGKFIRINFDVTGYIVGANIETYLLEKSRAVRQAKDERTFHIFYQLLSGAGEHLKSDLLLEGFNNYRFLSNGYIPIPGQQDKDNFQETMEAMHIMGFSHEEILSMLKVVSSVLQFGNISFKKERNTDQASMPENTVAQKLCHLLGMNVMEFTRAILTPRIKVGRDYVQKAQTKEQADFAVEALAKATYERLFRWLVHRINKALDRTKRQGASFIGILDIAGFEIFELNSFEQLCINYTNEKLQQLFNHTMFILEQEEYQREGIEWNFIDFGLDLQPCIDLIERPANPPGVLALLDEECWFPKATDKTFVEKLVQEQGSHSKFQKPRQLKDKADFCIIHYAGKVDYKADEWLMKNMDPLNDNVATLLHQSSDRFVAELWKDVDRIVGLDQVTGMTETAFGSAYKTKKGMFRTVGQLYKESLTKLMATLRNTNPNFVRCIIPNHEKRAGKLDPHLVLDQLRCNGVLEGIRICRQGFPNRIVFQEFRQRYEILTPNAIPKGFMDGKQACERMIRALELDPNLYRIGQSKIFFRAGVLAHLEEERDLKITDIIIFFQAVCRGYLARKAFAKKQQQLSALKVLQRNCAAYLKLRHWQWWRVFTKVKPLLQVTRQEEELQAKDEELLKVKEKQTKVEGELEEMERKHQQLLEEKNILAEQLQAETELFAEAEEMRARLAAKKQELEEILHDLESRVEEEEERNQILQNEKKKMQAHIQDLEEQLDEEEGARQKLQLEKVTAEAKIKKMEEEILLLEDQNSKFIKEKKLMEDRIAECSSQLAEEEEKAKNLAKIRNKQEVMISDLEERLKKEEKTRQELEKAKRKLDGETTDLQDQIAELQAQIDELKLQLAKKEEELQGALARGDDETLHKNNALKVVRELQAQIAELQEDFESEKASRNKAEKQKRDLSEELEALKTELEDTLDTTAAQQELRTKREQEVAELKKALEEETKNHEAQIQDMRQRHATALEELSEQLEQAKRFKANLEKNKQGLETDNKELACEVKVLQQVKAESEHKRKKLDAQVQELHAKVSEGDRLRVELAEKASKLQNELDNVSTLLEEAEKKGIKFAKDAASLESQLQDTQELLQEETRQKLNLSSRIRQLEEEKNSLQEQQEEEEEARKNLEKQVLALQSQLADTKKKVDDDLGTIESLEEAKKKLLKDAEALSQRLEEKALAYDKLEKTKNRLQQELDDLTVDLDHQRQVASNLEKKQKKFDQLLAEEKSISARYAEERDRAEAEAREKETKALSLARALEEALEAKEEFERQNKQLRADMEDLMSSKDDVGKNVHELEKSKRALEQQVEEMRTQLEELEDELQATEDAKLRLEVNMQAMKAQFERDLQTRDEQNEEKKRLLIKQVRELEAELEDERKQRALAVASKKKMEIDLKDLEAQIEAANKARDEVIKQLRKLQAQMKDYQRELEEARASRDEIFAQSKESEKKLKSLEAEILQLQEELASSERARRHAEQERDELADEITNSASGKSALLDEKRRLEARIAQLEEELEEEQSNMELLNDRFRKTTLQVDTLNAELAAERSAAQKSDNARQQLERQNKELKAKLQELEGAVKSKFKATISALEAKIGQLEEQLEQEAKERAAANKLVRRTEKKLKEIFMQVEDERRHADQYKEQMEKANARMKQLKRQLEEAEEEATRANASRRKLQRELDDATEANEGLSREVSTLKNRLRRGGPISFSSSRSGRRQLHLEGASLELSDDDTESKTSDVNETQPPQSE",
# ACTN2_HUMAN
"MNQIEPGVQYNYVYDEDEYMIQEEEWDRDLLLDPAWEKQQRKTFTAWCNSHLRKAGTQIENIEEDFRNGLKLMLLLEVISGERLPKPDRGKMRFHKIANVNKALDYIASKGVKLVSIGAEEIVDGNVKMTLGMIWTIILRFAIQDISVEETSAKEGLLLWCQRKTAPYRNVNIQNFHTSWKDGLGLCALIHRHRPDLIDYSKLNKDDPIGNINLAMEIAEKHLDIPKMLDAEDIVNTPKPDERAIMTYVSCFYHAFAGAEQAETAANRICKVLAVNQENERLMEEYERLASELLEWIRRTIPWLENRTPEKTMQAMQKKLEDFRDYRRKHKPPKVQEKCQLEINFNTLQTKLRISNRPAFMPSEGKMVSDIAGAWQRLEQAEKGYEEWLLNEIRRLERLEHLAEKFRQKASTHETWAYGKEQILLQKDYESASLTEVRALLRKHEAFESDLAAHQDRVEQIAAIAQELNELDYHDAVNVNDRCQKICDQWDRLGTLTQKRREALERMEKLLETIDQLHLEFAKRAAPFNNWMEGAMEDLQDMFIVHSIEEIQSLITAHEQFKATLPEADGERQSIMAIQNEVEKVIQSYNIRISSSNPYSTVTMDELRTKWDKVKQLVPIRDQSLQEELARQHANERLRRQFAAQANAIGPWIQNKMEEIARSSIQITGALEDQMNQLKQYEHNIINYKNNIDKLEGDHQLIQEALVFDNKHTNYTMEHIRVGWELLLTTIARTINEVETQILTRDAKGITQEQMNEFRASFNHFDRRKNGLMDHEDFRACLISMGYDLGEAEFARIMTLVDPNGQGTVTFQSFIDFMTRETADTDTAEQVIASFRILASDKPYILAEELRRELPPDQAQYCIKRMPAYSGPGSVPGALDYAAFSSALYGESDL",
# ADDA_HUMAN
"MNGDSRAAVVTSPPPTTAPHKERYFDRVDENNPEYLRERNMAPDLRQDFNMMEQKKRVSMILQSPAFCEELESMIQEQFKKGKNPTGLLALQQIADFMTTNVPNVYPAAPQGGMAALNMSLGMVTPVNDLRGSDSIAYDKGEKLLRCKLAAFYRLADLFGWSQLIYNHITTRVNSEQEHFLIVPFGLLYSEVTASSLVKINLQGDIVDRGSTNLGVNQAGFTLHSAIYAARPDVKCVVHIHTPAGAAVSAMKCGLLPISPEALSLGEVAYHDYHGILVDEEEKVLIQKNLGPKSKVLILRNHGLVSVGESVEEAFYYIHNLVVACEIQVRTLASAGGPDNLVLLNPEKYKAKSRSPGSPVGEGTGSPPKWQIGEQEFEALMRMLDNLGYRTGYPYRYPALREKSKKYSDVEVPASVTGYSFASDGDSGTCSPLRHSFQKQQREKTRWLNSGRGDEASEEGQNGSSPKSKTKWTKEDGHRTSTSAVPNLFVPLNTNPKEVQEMRNKIREQNLQDIKTAGPQSQVLCGVVMDRSLVQGELVTASKAIIEKEYQPHVIVSTTGPNPFTTLTDRELEEYRREVERKQKGSEENLDEAREQKEKSPPDQPAVPHPPPSTPIKLEEDLVPEPTTGDDSDAATFKPTLPDLSPDEPSEALGFPMLEKEEEAHRPPSPTEAPTEASPEPAPDPAPVAEEAAPSAVEEGAAADPGSDGSPGKSPSKKKKKFRTPSFLKKSKKKSDS",
# ADDB_HUMAN
"MSEETVPEAASPPPPQGQPYFDRFSEDDPEYMRLRNRAADLRQDFNLMEQKKRVTMILQSPSFREELEGLIQEQMKKGNNSSNIWALRQIADFMASTSHAVFPTSSMNVSMMTPINDLHTADSLNLAKGERLMRCKISSVYRLLDLYGWAQLSDTYVTLRVSKEQDHFLISPKGVSCSEVTASSLIKVNILGEVVEKGSSCFPVDTTGFCLHSAIYAARPDVRCIIHLHTPATAAVSAMKWGLLPVSHNALLVGDMAYYDFNGEMEQEADRINLQKCLGPTCKILVLRNHGVVALGDTVEEAFYKIFHLQAACEIQVSALSSAGGVENLILLEQEKHRPHEVGSVQWAGSTFGPMQKSRLGEHEFEALMRMLDNLGYRTGYTYRHPFVQEKTKHKSEVEIPATVTAFVFEEDGAPVPALRQHAQKQQKEKTRWLNTPNTYLRVNVADEVQRSMGSPRPKTTWMKADEVEKSSSGMPIRIENPNQFVPLYTDPQEVLEMRNKIREQNRQDVKSAGPQSQLLASVIAEKSRSPSTESQLMSKGDEDTKDDSEETVPNPFSQLTDQELEEYKKEVERKKLELDGEKETAPEEPGSPAKSAPASPVQSPAKEAETKSPLVSPSKSLEEGTKKTETSKAATTEPETTQPEGVVVNGREEEQTAEEILSKGLSQMTTSADTDVDTSKDKTESVTSGPMSPEGSPSKSPSKKKKKFRTPSFLKKSKKKEKVES",
# ARBK2_HUMAN
"MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKYLAERNEITFDKIFNQKIGFLLFKDFCLNEINEAVPQVKFYEEIKEYEKLDNEEDRLCRSRQIYDAYIMKELLSCSHPFSKQAVEHVQSHLSKKQVTSTLFQPYIEEICESLRGDIFQKFMESDKFTRFCQWKNVELNIHLTMNEFSVHRIIGRGGFGEVYGCRKADTGKMYAMKCLDKKRIKMKQGETLALNERIMLSLVSTGDCPFIVCMTYAFHTPDKLCFILDLMNGGDLHYHLSQHGVFSEKEMRFYATEIILGLEHMHNRFVVYRDLKPANILLDEHGHARISDLGLACDFSKKKPHASVGTHGYMAPEVLQKGTAYDSSADWFSLGCMLFKLLRGHSPFRQHKTKDKHEIDRMTLTVNVELPDTFSPELKSLLEGLLQRDVSKRLGCHGGGSQEVKEHSFFKGVDWQHVYLQKYPPPLIPPRGEVNAADAFDIGSFDEEDTKGIKLLDCDQELYKNFPLVISERWQQEVTETVYEAVNADTDKIEARKRAKNKQLGHEEDYALGKDCIMHGYMLKLGNPFLTQWQRRYFYLFPNRLEWRGEGESRQNLLTMEQILSVEETQIKDKKCILFRIKGGKQFVLQCESDPEFVQWKKELNETFKEAQRLLRRAPKFLNKPRSGTVELPKPSLCHRNSNGL",
# FUS_HUMAN
"MASNDYTQQATQSYGAYPTQPGQGYSQQSSQPYGQQSYSGYSQSTDTSGYGQSSYSSYGQSQNTGYGTQSTPQGYGSTGGYGSSQSSQSSYGQQSSYPGYGQQPAPSSTSGSYGSSSQSSSYGQPQSGSYSQQPSYGGQQQSYGQQQSYNPPQGYGQQNQYNSSSGGGGGGGGGGNYGQDQSSMSSGGGSGGGYGNQDQSGGGGSGGYGQQDRGGRGRGGSGGGGGGGGGGYNRSSGGYEPRGRGGGRGGRGGMGGSDRGGFNKFGGPRDQGSRHDSEQDNSDNNTIFVQGLGENVTIESVADYFKQIGIIKTNKKTGQPMINLYTDRETGKLKGEATVSFDDPPSAKAAIDWFDGKEFSGNPIKVSFATRRADFNRGGGNGRGGRGRGGPMGRGGYGGGGSGGGGRGGFPSGGGGGGGQQRAGDWKCPNPTCENMNFSWRNECNQCKAPKPDGPGGGPGGSHMGGNYGDDRRGGRGGYDRGGYRGRGGDRGGFRGGRGGGDRGGFGPGKMDSRGEHRQDRRERPY",
# VATE1_HUMAN
"MALSDADVQKQIKHMMAFIEQEANEKAEEIDAKAEEEFNIEKGRLVQTQRLKIMEYYEKKEKQIEQQKKIQMSNLMNQARLKVLRARDDLITDLLNEAKQRLSKVVKDTTRYQVLLDGLVLQGLYQLLEPRMIVRCRKQDFPLVKAAVQKAIPMYKIATKNDVDVQIDQESYLPEDIAGGVEIYNGDRKIKVSNTLESRLDLIAQQMMPEVRGALFGANANRKFLD",
# ACHA7_HUMAN
"MRCSPGGVWLALAASLLHVSLQGEFQRKLYKELVKNYNPLERPVANDSQPLTVYFSLSLLQIMDVDEKNQVLTTNIWLQMSWTDHYLQWNVSEYPGVKTVRFPDGQIWKPDILLYNSADERFDATFHTNVLVNSSGHCQYLPPGIFKSSCYIDVRWFPFDVQHCKLKFGSWSYGGWSLDLQMQEADISGYIPNGEWDLVGIPGKRSERFYECCKEPYPDVTFTVTMRRRTLYYGLNLLIPCVLISALALLVFLLPADSGEKISLGITVLLSLTVFMLLVAEIMPATSDSVPLIAQYFASTMIIVGLSVVVTVIVLQYHHHDPDGGKMPKWTRVILLNWCAWFLRMKRPGEDKVRPACQHKQRRCSLASVEMSAVAPPPASNGNLLYIGFRGLDGVHCVPTPDSGVVCGRMACSPTHDEHLLHGGQPPEGDPDLAKILEEVRYIANRFRCQDESEAVCSEWKFAACVVDRLCLMAFSVFTIICTIGILMSAPNFVEAVSKDFA",
# RL4_HUMAN
"MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRKNNRQPYAVSELAGHQTSAESWGTGRAVARIPRVRGGGTHRSGQGAFGNMCRGGRMFAPTKTWRRWHRRVNTTQKRYAICSALAASALPALVMSKGHRIEEVPELPLVVEDKVEGYKKTKEAVLLLKKLKAWNDIKKVYASQRMRAGKGKMRNRRRIQRRGPCIIYNEDNGIIKAFRNIPGITLLNVSKLNILKLAPGGHVGRFCIWTESAFRKLDELYGTWRKAASLKSNYNLPMHKMINTDLSRILKSPEIQRALRAPRKKIHRRVLKKNPLKNLRIMLKLNPYAKTMRRNTILRQARNHKLRVDKAAAAAAALQAKSDEKAAVAGKKPVVGKKGKKAAVGVKKQKKPLVGKKAAATKKPAPEKKPAEKKPTTEEKKPAA",
# PP1G_HUMAN
"MADLDKLNIDSIIQRLLEVRGSKPGKNVQLQENEIRGLCLKSREIFLSQPILLELEAPLKICGDIHGQYYDLLRLFEYGGFPPESNYLFLGDYVDRGKQSLETICLLLAYKIKYPENFFLLRGNHECASINRIYGFYDECKRRYNIKLWKTFTDCFNCLPIAAIVDEKIFCCHGGLSPDLQSMEQIRRIMRPTDVPDQGLLCDLLWSDPDKDVLGWGENDRGVSFTFGAEVVAKFLHKHDLDLICRAHQVVEDGYEFFAKRQLVTLFSAPNYCGEFDNAGAMMSVDETLMCSFQILKPAEKKKPNATRPVTPPRGMITKQAKK",
# SYUA_HUMAN
"MDVFMKGLSKAKEGVVAAAEKTKQGVAEAAGKTKEGVLYVGSKTKEGVVHGVATVAEKTKEQVTNVGGAVVTGVTAVAQKTVEGAGSIAAATGFVKKDQLGKNEEGAPQEGILEDMPVDPDNEAYEMPSEEGYQDYEPEA",
# RBMX_HUMAN
"MVEADRPGKLFIGGLNTETNEKALEAVFGKYGRIVEVLLMKDRETNKSRGFAFVTFESPADAKDAARDMNGKSLDGKAIKVEQATKPSFESGRRGPPPPPRSRGPPRGLRGGRGGSGGTRGPPSRGGHMDDGGYSMNFNMSSSRGPLPVKRGPPPRSGGPPPKRSAPSGPVRSSSGMGGRAPVSRGRDSYGGPPRREPLPSRRDVYLSPRDDGYSTKDSYSSRDYPSSRDTRDYAPPPRDYTYRDYGHSSSRDDYPSRGYSDRDGYGRDRDYSDHPSGGSYRDSYESYGNSRSAPPTRGPPPSYGGSSRYDDYSSSRDGYGGSRDSYSSSRSDLYSSGRDRVGRQERGLPPSMERGYPPPRDSYSSSSRGAPRGGGRGGSRSDRGGGRSRY",
# GNAL_HUMAN
"MGCLGGNSKTTEDQGVDEKERREANKKIEKQLQKERLAYKATHRLLLLGAGESGKSTIVKQMRILHVNGFNPEEKKQKILDIRKNVKDAIVTIVSAMSTIIPPVPLANPENQFRSDYIKSIAPITDFEYSQEFFDHVKKLWDDEGVKACFERSNEYQLIDCAQYFLERIDSVSLVDYTPTDQDLLRCRVLTSGIFETRFQVDKVNFHMFDVGGQRDERRKWIQCFNDVTAIIYVAACSSYNMVIREDNNTNRLRESLDLFESIWNNRWLRTISIILFLNKQDMLAEKVLAGKSKIEDYFPEYANYTVPEDATPDAGEDPKVTRAKFFIRDLFLRISTATGDGKHYCYPHFTCAVDTENIRRVFNDCRDIIQRMHLKQYELL",
# VATA_HUMAN
"MDFSKLPKILDEDKESTFGYVHGVSGPVVTACDMAGAAMYELVRVGHSELVGEIIRLEGDMATIQVYEETSGVSVGDPVLRTGKPLSVELGPGIMGAIFDGIQRPLSDISSQTQSIYIPRGVNVSALSRDIKWDFTPCKNLRVGSHITGGDIYGIVSENSLIKHKIMLPPRNRGTVTYIAPPGNYDTSDVVLELEFEGVKEKFTMVQVWPVRQVRPVTEKLPANHPLLTGQRVLDALFPCVQGGTTAIPGAFGCGKTVISQSLSKYSNSDVIIYVGCGERGNEMSEVLRDFPELTMEVDGKVESIMKRTALVANTSNMPVAAREASIYTGITLSEYFRDMGYHVSMMADSTSRWAEALREISGRLAEMPADSGYPAYLGARLASFYERAGRVKCLGNPEREGSVSIVGAVSPPGGDFSDPVTSATLGIVQVFWGLDKKLAQRKHFPSVNWLISYSKYMRALDEYYDKHFTEFVPLRTKAKEILQEEEDLAEIVQLVGKASLAETDKITLEVAKLIKDDFLQQNGYTPYDRFCPFYKTVGMLSNMIAFYDMARRAVETTAQSDNKITWSIIREHMGDILYKLSSMKFKDPLKDGEAKIKSDYAQLLEDMQNAFRSLED",
# IF4A3_HUMAN
"MATTATMATSGSARKRLLKEEDMTKVEFETSEEVDVTPTFDTMGLREDLLRGIYAYGFEKPSAIQQRAIKQIIKGRDVIAQSQSGTGKTATFSISVLQCLDIQVRETQALILAPTRELAVQIQKGLLALGDYMNVQCHACIGGTNVGEDIRKLDYGQHVVAGTPGRVFDMIRRRSLRTRAIKMLVLDEADEMLNKGFKEQIYDVYRYLPPATQVVLISATLPHEILEMTNKFMTDPIRILVKRDELTLEGIKQFFVAVEREEWKFDTLCDLYDTLTITQAVIFCNTKRKVDWLTEKMREANFTVSSMHGDMPQKERESIMKEFRSGASRVLISTDVWARGLDVPQVSLIINYDLPNNRELYIHRIGRSGRYGRKGVAINFVKNDDIRILRDIEQYYSTQIDEMPMNVADLI",
# RS19_HUMAN
"MPGVTVKDVNQQEFVRALAAFLKKSGKLKVPEWVDTVKLAKHKELAPYDENWFYTRAASTARHLYLRGGAGVGSMTKIYGGRQRNGVMPSHFSRGSKSVARRVLQALEGLKMVEKDQDGGRKLTPQGQRDLDRIAGQVAAANKKH",
# RL3_HUMAN
"MSHRKFSAPRHGSLGFLPRKRSSRHRGKVKSFPKDDPSKPVHLTAFLGYKAGMTHIVREVDRPGSKVNKKEVVEAVTIVETPPMVVVGIVGYVETPRGLRTFKTVFAEHISDECKRRFYKNWHKSKKKAFTKYCKKWQDEDGKKQLEKDFSSMKKYCQVIRVIAHTQMRLLPLRQKKAHLMEIQVNGGTVAEKLDWARERLEQQVPVNQVFGQDEMIDVIGVTKGKGYKGVTSRWHTKKLPRKTHRGLRKVACIGAWHPARVAFSVARAGQKGYHHRTEINKKIYKIGQGYLIKDGKLIKNNASTDYDLSDKSINPLGGFVHYGEVTNDFVMLKGCVVGTKKRVLTLRKSLLVQTKRRALEKIDLKFIDTTSKFGHGRFQTMEEKKAFMGPLKKDRIAKEEGA",
# GRIK1_HUMAN
"MEHGTLLAQPGLWTRDTSWALLYFLCYILPQTAPQVLRIGGIFETVENEPVNVEELAFKFAVTSINRNRTLMPNTTLTYDIQRINLFDSFEASRRACDQLALGVAALFGPSHSSSVSAVQSICNALEVPHIQTRWKHPSVDNKDLFYINLYPDYAAISRAILDLVLYYNWKTVTVVYEDSTGLIRLQELIKAPSRYNIKIKIRQLPSGNKDAKPLLKEMKKGKEFYVIFDCSHETAAEILKQILFMGMMTEYYHYFFTTLDLFALDLELYRYSGVNMTGFRLLNIDNPHVSSIIEKWSMERLQAPPRPETGLLDGMMTTEAALMYDAVYMVAIASHRASQLTVSSLQCHRHKPWRLGPRFMNLIKEARWDGLTGHITFNKTNGLRKDFDLDIISLKEEGTEKAAGEVSKHLYKVWKKIGIWNSNSGLNMTDSNKDKSSNITDSLANRTLIVTTILEEPYVMYRKSDKPLYGNDRFEGYCLDLLKELSNILGFIYDVKLVPDGKYGAQNDKGEWNGMVKELIDHRADLAVAPLTITYVREKVIDFSKPFMTLGISILYRKPNGTNPGVFSFLNPLSPDIWMYVLLACLGVSCVLFVIARFTPYEWYNPHPCNPDSDVVENNFTLLNSFWFGVGALMQQGSELMPKALSTRIVGGIWWFFTLIIISSYTANLAAFLTVERMESPIDSADDLAKQTKIEYGAVRDGSTMTFFKKSKISTYEKMWAFMSSRQQTALVRNSDEGIQRVLTTDYALLMESTSIEYVTQRNCNLTQIGGLIDSKGYGVGTPIGSPYRDKITIAILQLQEEGKLHMMKEKWWRGNGCPEEDNKEASALGVENIGGIFIVLAAGLVLSVFVAIGEFIYKSRKNNDIEQAFCFFYGLQCKQTHPTNSTSGTTLSTDLECGKLIREERGIRKQSSVHTV",
# CAP2_HUMAN
"MANMQGLVERLERAVSRLESLSAESHRPPGNCGEVNGVIAGVAPSVEAFDKLMDSMVAEFLKNSRILAGDVETHAEMVHSAFQAQRAFLLMASQYQQPHENDVAALLKPISEKIQEIQTFRERNRGSNMFNHLSAVSESIPALGWIAVSPKPGPYVKEMNDAATFYTNRVLKDYKHSDLRHVDWVKSYLNIWSELQAYIKEHHTTGLTWSKTGPVASTVSAFSVLSSGPGLPPPPPPLPPPGPPPLFENEGKKEESSPSRSALFAQLNQGEAITKGLRHVTDDQKTYKNPSLRAQGGQTQSPTKSHTPSPTSPKSYPSQKHAPVLELEGKKWRVEYQEDRNDLVISETELKQVAYIFKCEKSTIQIKGKVNSIIIDNCKKLGLVFDNVVGIVEVINSQDIQIQVMGRVPTISINKTEGCHIYLSEDALDCEIVSAKSSEMNILIPQDGDYREFPIPEQFKTAWDGSKLITEPAEIMA",
# ADCY8_HUMAN
"MELSDVRCLTGSEELYTIHPTPPAGDGRSASRPQRLLWQTAVRHITEQRFIHGHRGGSGSGSGGSGKASDPAGGGPNHHAPQLSGDSALPLYSLGPGERAHSTCGTKVFPERSGSGSASGSGGGGDLGFLHLDCAPSNSDFFLNGGYSYRGVIFPTLRNSFKSRDLERLYQRYFLGQRRKSEVVMNVLDVLTKLTLLVLHLSLASAPMDPLKGILLGFFTGIEVVICALVVVRKDTTSHTYLQYSGVVTWVAMTTQILAAGLGYGLLGDGIGYVLFTLFATYSMLPLPLTWAILAGLGTSLLQVILQVVIPRLAVISINQVVAQAVLFMCMNTAGIFISYLSDRAQRQAFLETRRCVEARLRLETENQRQERLVLSVLPRFVVLEMINDMTNVEDEHLQHQFHRIYIHRYENVSILFADVKGFTNLSTTLSAQELVRMLNELFARFDRLAHEHHCLRIKILGDCYYCVSGLPEPRQDHAHCCVEMGLSMIKTIRYVRSRTKHDVDMRIGIHSGSVLCGVLGLRKWQFDVWSWDVDIANKLESGGIPGRIHISKATLDCLNGDYNVEEGHGKERNEFLRKHNIETYLIKQPEDSLLSLPEDIVKESVSSSDRRNSGATFTEGSWSPELPFDNIVGKQNTLAALTRNSINLLPNHLAQALHVQSGPEEINKRIEHTIDLRSGDKLRREHIKPFSLMFKDSSLEHKYSQMRDEVFKSNLVCAFIVLLFITAIQSLLPSSRVMPMTIQFSILIMLHSALVLITTAEDYKCLPLILRKTCCWINETYLARNVIIFASILINFLGAILNILWCDFDKSIPLKNLTFNSSAVFTDICSYPEYFVFTGVLAMVTCAVFLRLNSVLKLAVLLIMIAIYALLTETVYAGLFLRYDNLNHSGEDFLGTKEVSLLLMAMFLLAVFYHGQQLEYTARLDFLWRVQAKEEINEMKELREHNENMLRNILPSHVARHFLEKDRDNEELYSQSYDAVGVMFASIPGFADFYSQTEMNNQGVECLRLLNEIIADFDELLGEDRFQDIEKIKTIGSTYMAVSGLSPEKQQCEDKWGHLCALADFSLALTESIQEINKHSFNNFELRIGISHGSVVAGVIGAKKPQYDIWGKTVNLASRMDSTGVSGRIQVPEETYLILKDQGFAFDYRGEIYVKGISEQEGKIKTYFLLGRVQPNPFILPPRRLPGQYSLAAVVLGLVQSLNRQRQKQLLNENNNTGIIKGHYNRRTLLSPSGTEPGAQAEGTDKSDLP",
# VHL_HUMAN
"MPRRAENWDEAEVGAEEAGVEEYGPEEDGGEESGAEESGPEESGPEELGAEEEMEAGRPRPVLRSVNSREPSQVIFCNRSPRVVLPVWLNFDGEPQPYPTLPPGTGRRIHSYRGHLWLFRDAGTHDGLLVNQTELFVPSLNVDGQPIFANITLPVYTLKERCLQVVRSLVKPENYRRLDIVRSLYEDLEDHPNVQKDLERLTQERIAHQRMGD",
# RL13A_HUMAN
"MAEVQVLVLDGRGHLLGRLAAIVAKQVLLGRKVVVVRCEGINISGNFYRNKLKYLAFLRKRMNTNPSRGPYHFRAPSRIFWRTVRGMLPHKTKRGQAALDRLKVFDGIPPPYDKKKRMVVPAALKVVRLKPTRKFAYLGRLAHEVGWKYQAVTATLEEKRKEKAKIHYRKKKQLMRLRKQAEKNVEKKIDKYTEVLKTHGLLV",
# STAT3_HUMAN
"MAQWNQLQQLDTRYLEQLHQLYSDSFPMELRQFLAPWIESQDWAYAASKESHATLVFHNLLGEIDQQYSRFLQESNVLYQHNLRRIKQFLQSRYLEKPMEIARIVARCLWEESRLLQTAATAAQQGGQANHPTAAVVTEKQQMLEQHLQDVRKRVQDLEQKMKVVENLQDDFDFNYKTLKSQGDMQDLNGNNQSVTRQKMQQLEQMLTALDQMRRSIVSELAGLLSAMEYVQKTLTDEELADWKRRQQIACIGGPPNICLDRLENWITSLAESQLQTRQQIKKLEELQQKVSYKGDPIVQHRPMLEERIVELFRNLMKSAFVVERQPCMPMHPDRPLVIKTGVQFTTKVRLLVKFPELNYQLKIKVCIDKDSGDVAALRGSRKFNILGTNTKVMNMEESNNGSLSAEFKHLTLREQRCGNGGRANCDASLIVTEELHLITFETEVYHQGLKIDLETHSLPVVVISNICQMPNAWASILWYNMLTNNPKNVNFFTKPPIGTWDQVAEVLSWQFSSTTKRGLSIEQLTTLAEKLLGPGVNYSGCQITWAKFCKENMAGKGFSFWVWLDNIIDLVKKYILALWNEGYIMGFISKERERAILSTKPPGTFLLRFSESSKEGGVTFTWVEKDISGKTQIQSVEPYTKQQLNNMSFAEIIMGYKIMDATNILVSPLVYLYPDIPKEEAFGKYCRPESQEHPEADPGSAAPYLKTKFICVTPTTCSNTIDLPMSPRTLDSLMQFGNNGEGAEPSAGGQFESLTFDMELTSECATSPM",
# UBP8_HUMAN
"MPAVASVPKELYLSSSLKDLNKKTEVKPEKISTKSYVHSALKIFKTAEECRLDRDEERAYVLYMKYVTVYNLIKKRPDFKQQQDYFHSILGPGNIKKAVEEAERLSESLKLRYEEAEVRKKLEEKDRQEEAQRLQQKRQETGREDGGTLAKGSLENVLDSKDKTQKSNGEKNEKCETKEKGAITAKELYTMMTDKNISLIIMDARRMQDYQDSCILHSLSVPEEAISPGVTASWIEAHLPDDSKDTWKKRGNVEYVVLLDWFSSAKDLQIGTTLRSLKDALFKWESKTVLRNEPLVLEGGYENWLLCYPQYTTNAKVTPPPRRQNEEVSISLDFTYPSLEESIPSKPAAQTPPASIEVDENIELISGQNERMGPLNISTPVEPVAASKSDVSPIIQPVPSIKNVPQIDRTKKPAVKLPEEHRIKSESTNHEQQSPQSGKVIPDRSTKPVVFSPTLMLTDEEKARIHAETALLMEKNKQEKELRERQQEEQKEKLRKEEQEQKAKKKQEAEENEITEKQQKAKEEMEKKESEQAKKEDKETSAKRGKEITGVKRQSKSEHETSDAKKSVEDRGKRCPTPEIQKKSTGDVPHTSVTGDSGSGKPFKIKGQPESGILRTGTFREDTDDTERNKAQREPLTRARSEEMGRIVPGLPSGWAKFLDPITGTFRYYHSPTNTVHMYPPEMAPSSAPPSTPPTHKAKPQIPAERDREPSKLKRSYSSPDITQAIQEEEKRKPTVTPTVNRENKPTCYPKAEISRLSASQIRNLNPVFGGSGPALTGLRNLGNTCYMNSILQCLCNAPHLADYFNRNCYQDDINRSNLLGHKGEVAEEFGIIMKALWTGQYRYISPKDFKITIGKINDQFAGYSQQDSQELLLFLMDGLHEDLNKADNRKRYKEENNDHLDDFKAAEHAWQKHKQLNESIIVALFQGQFKSTVQCLTCHKKSRTFEAFMYLSLPLASTSKCTLQDCLRLFSKEEKLTDNNRFYCSHCRARRDSLKKIEIWKLPPVLLVHLKRFSYDGRWKQKLQTSVDFPLENLDLSQYVIGPKNNLKKYNLFSVSNHYGGLDGGHYTAYCKNAARQRWFKFDDHEVSDISVSSVKSSAAYILFYTSLGPRVTDVAT",
# IF2G_HUMAN
"MAGGEAGVTLGQPHLSRQDLTTLDVTKLTPLSHEVISRQATINIGTIGHVAHGKSTVVKAISGVHTVRFKNELERNITIKLGYANAKIYKLDDPSCPRPECYRSCGSSTPDEFPTDIPGTKGNFKLVRHVSFVDCPGHDILMATMLNGAAVMDAALLLIAGNESCPQPQTSEHLAAIEIMKLKHILILQNKIDLVKESQAKEQYEQILAFVQGTVAEGAPIIPISAQLKYNIEVVCEYIVKKIPVPPRDFTSEPRLIVIRSFDVNKPGCEVDDLKGGVAGGSILKGVLKVGQEIEVRPGIVSKDSEGKLMCKPIFSKIVSLFAEHNDLQYAAPGGLIGVGTKIDPTLCRADRMVGQVLGAVGALPEIFTELEISYFLLRRLLGVRTEGDKKAAKVQKLSKNEVLMVNIGSLSTGGRVSAVKADLGKIVLTNPVCTEVGEKIALSRRVEKHWRLIGWGQIRRGVTIKPTVDDD",
# OPRD_HUMAN
"MEPAPSAGAELQPPLFANASDAYPSACPSAGANASGPPGARSASSLALAIAITALYSAVCAVGLLGNVLVMFGIVRYTKMKTATNIYIFNLALADALATSTLPFQSAKYLMETWPFGELLCKAVLSIDYYNMFTSIFTLTMMSVDRYIAVCHPVKALDFRTPAKAKLINICIWVLASGVGVPIMVMAVTRPRDGAVVCMLQFPSPSWYWDTVTKICVFLFAFVVPILIITVCYGLMLLRLRSVRLLSGSKEKDRSLRRITRMVLVVVGAFVVCWAPIHIFVIVWTLVDIDRRDPLVVAALHLCIALGYANSSLNPVLYAFLDENFKRCFRQLCRKPCGRPDPSSFSRAREATARERVTACTPSDGPGGGAAA",
# OPRK_HUMAN
"MDSPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSEDAQLEPAHISPAIPVIITAVYSVVFVVGLVGNSLVMFVIIRYTKMKTATNIYIFNLALADALVTTTMPFQSTVYLMNSWPFGDVLCKIVISIDYYNMFTSIFTLTMMSVDRYIAVCHPVKALDFRTPLKAKIINICIWLLSSSVGISAIVLGGTKVREDVDVIECSLQFPDDDYSWWDLFMKICVFIFAFVIPVLIIIVCYTLMILRLKSVRLLSGSREKDRNLRRITRLVLVVVAVFVVCWTPIHIFILVEALGSTSHSTAALSSYYFCIALGYTNSSLNPILYAFLDENFKRCFRDFCFPLKMRMERQSTSRVRNTVQDPAYLRDIDGMNKPV",
# CASR_HUMAN
"MAFYSCCWVLLALTWHTSAYGPDQRAQKKGDIILGGLFPIHFGVAAKDQDLKSRPESVECIRYNFRGFRWLQAMIFAIEEINSSPALLPNLTLGYRIFDTCNTVSKALEATLSFVAQNKIDSLNLDEFCNCSEHIPSTIAVVGATGSGVSTAVANLLGLFYIPQVSYASSSRLLSNKNQFKSFLRTIPNDEHQATAMADIIEYFRWNWVGTIAADDDYGRPGIEKFREEAEERDICIDFSELISQYSDEEEIQHVVEVIQNSTAKVIVVFSSGPDLEPLIKEIVRRNITGKIWLASEAWASSSLIAMPQYFHVVGGTIGFALKAGQIPGFREFLKKVHPRKSVHNGFAKEFWEETFNCHLQEGAKGPLPVDTFLRGHEESGDRFSNSSTAFRPLCTGDENISSVETPYIDYTHLRISYNVYLAVYSIAHALQDIYTCLPGRGLFTNGSCADIKKVEAWQVLKHLRHLNFTNNMGEQVTFDECGDLVGNYSIINWHLSPEDGSIVFKEVGYYNVYAKKGERLFINEEKILWSGFSREVPFSNCSRDCLAGTRKGIIEGEPTCCFECVECPDGEYSDETDASACNKCPDDFWSNENHTSCIAKEIEFLSWTEPFGIALTLFAVLGIFLTAFVLGVFIKFRNTPIVKATNRELSYLLLFSLLCCFSSSLFFIGEPQDWTCRLRQPAFGISFVLCISCILVKTNRVLLVFEAKIPTSFHRKWWGLNLQFLLVFLCTFMQIVICVIWLYTAPPSSYRNQELEDEIIFITCHEGSLMALGFLIGYTCLLAAICFFFAFKSRKLPENFNEAKFITFSMLIFFIVWISFIPAYASTYGKFVSAVEVIAILAASFGLLACIFFNKIYIILFKPSRNTIEEVRCSTAAHAFKVAARATLRRSNVSRKRSSSLGGSTGSTPSSSISSKSNSEDPFPQPERQKQQQPLALTQQEQQQQPLTLPQQQRSQQQPRCKQKVIFGSGTVTFSLSFDEPQKNAMAHRNSTHQNSLEAQKSSDTLTRHEPLLPLQCGETDLDLTVQETGLQGPVGGDQRPEVEDPEELSPALVVSSSQSFVISGGGSTVTENVVNS",
# OX2G_HUMAN
"MERLVIRMPFSHLSTYSLVWVMAAVVLCTAQVQVVTQDEREQLYTPASLKCSLQNAQEALIVTWQKKKAVSPENMVTFSENHGVVIQPAYKDKINITQLGLQNSTITFWNITLEDEGCYMCLFNTFGFGKISGTACLTVYVQPIVSLHYKFSEDHLNITCSATARPAPMVFWKVPRSGIENSTVTLSHPNGTTSVTSILHIKDPKNQVGKEVICQVLHLGTVTDFKQTVNKGYWFSVPLLLSIVSLVILLVLISILLYWKRHRNQDRGELSQGVQKMT",
# WNT5A_HUMAN
"MKKSIGILSPGVALGMAGSAMSSKFFLVALAIFFSFAQVVIEANSWWSLGMNNPVQMSEVYIIGAQPLCSQLAGLSQGQKKLCHLYQDHMQYIGEGAKTGIKECQYQFRHRRWNCSTVDNTSVFGRVMQIGSRETAFTYAVSAAGVVNAMSRACREGELSTCGCSRAARPKDLPRDWLWGGCGDNIDYGYRFAKEFVDARERERIHAKGSYESARILMNLHNNEAGRRTVYNLADVACKCHGVSGSCSLKTCWLQLADFRKVGDALKEKYDSAAAMRLNSRGKLVQVNSRFNSPTTQDLVYIDPSPDYCVRNESTGSLGTQGRLCNKTSEGMDGCELMCCGRGYDQFKTVQTERCHCKFHWCCYVKCKKCTEIVDQFVCK",
# P2RY2_HUMAN
"MAADLGPWNDTINGTWDGDELGYRCRFNEDFKYVLLPVSYGVVCVPGLCLNAVALYIFLCRLKTWNASTTYMFHLAVSDALYAASLPLLVYYYARGDHWPFSTVLCKLVRFLFYTNLYCSILFLTCISVHRCLGVLRPLRSLRWGRARYARRVAGAVWVLVLACQAPVLYFVTTSARGGRVTCHDTSAPELFSRFVAYSSVMLGLLFAVPFAVILVCYVLMARRLLKPAYGTSGGLPRAKRKSVRTIAVVLAVFALCFLPFHVTRTLYYSFRSLDLSCHTLNAINMAYKVTRPLASANSCLDPVLYFLAGQRLVRFARDAKPPTGPSPATPARRRLGLRRSDRTDMQRIEDVLGSSEDSRRTESTPAGSENTKDIRL",
# S19A1_HUMAN
"MVPSSPAVEKQVPVEPGPDPELRSWRHLVCYLCFYGFMAQIRPGESFITPYLLGPDKNFTREQVTNEITPVLSYSYLAVLVPVFLLTDYLRYTPVLLLQGLSFVSVWLLLLLGHSVAHMQLMELFYSVTMAARIAYSSYIFSLVRPARYQRVAGYSRAAVLLGVFTSSVLGQLLVTVGRVSFSTLNYISLAFLTFSVVLALFLKRPKRSLFFNRDDRGRCETSASELERMNPGPGGKLGHALRVACGDSVLARMLRELGDSLRRPQLRLWSLWWVFNSAGYYLVVYYVHILWNEVDPTTNSARVYNGAADAASTLLGAITSFAAGFVKIRWARWSKLLIAGVTATQAGLVFLLAHTRHPSSIWLCYAAFVLFRGSYQFLVPIATFQIASSLSKELCALVFGVNTFFATIVKTIITFIVSDVRGLGLPVRKQFQLYSVYFLILSIIYFLGAMLDGLRHCQRGHHPRQPPAQGLRSAAEEKAAQALSVQDKGLGGLQPAQSPPLSPEDSLGAVGPASLEQRQSDPYLAQAPAPQAAEFLSPVTTPSPCTLCSAQASGPEAADETCPQLAVHPPGVSKLGLQCLPSDGVQNVNQ",
# GRM5_HUMAN
"MVLLLILSVLLLKEDVRGSAQSSERRVVAHMPGDIIIGALFSVHHQPTVDKVHERKCGAVREQYGIQRVEAMLHTLERINSDPTLLPNITLGCEIRDSCWHSAVALEQSIEFIRDSLISSEEEEGLVRCVDGSSSSFRSKKPIVGVIGPGSSSVAIQVQNLLQLFNIPQIAYSATSMDLSDKTLFKYFMRVVPSDAQQARAMVDIVKRYNWTYVSAVHTEGNYGESGMEAFKDMSAKEGICIAHSYKIYSNAGEQSFDKLLKKLTSHLPKARVVACFCEGMTVRGLLMAMRRLGLAGEFLLLGSDGWADRYDVTDGYQREAVGGITIKLQSPDVKWFDDYYLKLRPETNHRNPWFQEFWQHRFQCRLEGFPQENSKYNKTCNSSLTLKTHHVQDSKMGFVINAIYSMAYGLHNMQMSLCPGYAGLCDAMKPIDGRKLLESLMKTNFTGVSGDTILFDENGDSPGRYEIMNFKEMGKDYFDYINVGSWDNGELKMDDDEVWSKKSNIIRSVCSEPCEKGQIKVIRKGEVSCCWTCTPCKENEYVFDEYTCKACQLGSWPTDDLTGCDLIPVQYLRWGDPEPIAAVVFACLGLLATLFVTVVFIIYRDTPVVKSSSRELCYIILAGICLGYLCTFCLIAKPKQIYCYLQRIGIGLSPAMSYSALVTKTNRIARILAGSKKKICTKKPRFMSACAQLVIAFILICIQLGIIVALFIMEPPDIMHDYPSIREVYLICNTTNLGVVTPLGYNGLLILSCTFYAFKTRNVPANFNEAKYIAFTMYTTCIIWLAFVPIYFGSNYKIITMCFSVSLSATVALGCMFVPKVYIILAKPERNVRSAFTTSTVVRMHVGDGKSSSAASRSSSLVNLWKRRGSSGETLRYKDRRLAQHKSEIECFTPKGSMGNGGRATMSSSNGKSVTWAQNEKSSRGQHLWQRLSIHINKKENPNQTAVIKPFPKSTESRGLGAGAGAGGSAGGVGATGGAGCAGAGPGGPESPDAGPKALYDVAEAEEHFPAPARPRSPSPISTLSHRAGSASRTDDDVPSLHSEPVARSSSSQGSLMEQISSVVTRFTANISELNSMMLSTAAPSPGVGAPLCSSYLIPKEIQLPTTMTTFAEIQPLPAIEVTGGAQPAAGAQAAGDAARESPAAGPEAAAAKPDLEELVALTPPSPFRDSVDSGSTTPNSPVSESALCIPSSPKYDTLIIRDYTQSSSSL",
# TSN7_HUMAN
"MASRRMETKPVITCLKTLLIIYSFVFWITGVILLAVGVWGKLTLGTYISLIAENSTNAPYVLIGTGTTIVVFGLFGCFATCRGSPWMLKLYAMFLSLVFLAELVAGISGFVFRHEIKDTFLRTYTDAMQTYNGNDERSRAVDHVQRSLSCCGVQNYTNWSTSPYFLEHGIPPSCCMNETDCNPQDLHNLTVAATKVNQKGCYDLVTSFMETNMGIIAGVAFGIAFSQLIGMLLACCLSRFITANQYEMV",
# KPCI_HUMAN
"MPTQRDSSTMSHTVAGGGSGDHSHQVRVKAYYRGDIMITHFEPSISFEGLCNEVRDMCSFDNEQLFTMKWIDEEGDPCTVSSQLELEEAFRLYELNKDSELLIHVFPCVPERPGMPCPGEDKSIYRRGARRWRKLYCANGHTFQAKRFNRRAHCAICTDRIWGLGRQGYKCINCKLLVHKKCHKLVTIECGRHSLPQEPVMPMDQSSMHSDHAQTVIPYNPSSHESLDQVGEEKEAMNTRESGKASSSLGLQDFDLLRVIGRGSYAKVLLVRLKKTDRIYAMKVVKKELVNDDEDIDWVQTEKHVFEQASNHPFLVGLHSCFQTESRLFFVIEYVNGGDLMFHMQRQRKLPEEHARFYSAEISLALNYLHERGIIYRDLKLDNVLLDSEGHIKLTDYGMCKEGLRPGDTTSTFCGTPNYIAPEILRGEDYGFSVDWWALGVLMFEMMAGRSPFDIVGSSDNPDQNTEDYLFQVILEKQIRIPRSLSVKAASVLKSFLNKDPKERLGCHPQTGFADIQGHPFFRNVDWDMMEQKQVVPPFKPNISGEFGLDNFDSQFTNEPVQLTPDDDDIVRKIDQSEFEGFEYINPLLMSAEECV",
# GRIA1_HUMAN
"MQHIFAFFCTGFLGAVVGANFPNNIQIGGLFPNQQSQEHAAFRFALSQLTEPPKLLPQIDIVNISDSFEMTYRFCSQFSKGVYAIFGFYERRTVNMLTSFCGALHVCFITPSFPVDTSNQFVLQLRPELQDALISIIDHYKWQKFVYIYDADRGLSVLQKVLDTAAEKNWQVTAVNILTTTEEGYRMLFQDLEKKKERLVVVDCESERLNAILGQIIKLEKNGIGYHYILANLGFMDIDLNKFKESGANVTGFQLVNYTDTIPAKIMQQWKNSDARDHTRVDWKRPKYTSALTYDGVKVMAEAFQSLRRQRIDISRRGNAGDCLANPAVPWGQGIDIQRALQQVRFEGLTGNVQFNEKGRRTNYTLHVIEMKHDGIRKIGYWNEDDKFVPAATDAQAGGDNSSVQNRTYIVTTILEDPYVMLKKNANQFEGNDRYEGYCVELAAEIAKHVGYSYRLEIVSDGKYGARDPDTKAWNGMVGELVYGRADVAVAPLTITLVREEVIDFSKPFMSLGISIMIKKPQKSKPGVFSFLDPLAYEIWMCIVFAYIGVSVVLFLVSRFSPYEWHSEEFEEGRDQTTSDQSNEFGIFNSLWFSLGAFMQQGCDISPRSLSGRIVGGVWWFFTLIIISSYTANLAAFLTVERMVSPIESAEDLAKQTEIAYGTLEAGSTKEFFRRSKIAVFEKMWTYMKSAEPSVFVRTTEEGMIRVRKSKGKYAYLLESTMNEYIEQRKPCDTMKVGGNLDSKGYGIATPKGSALRNPVNLAVLKLNEQGLLDKLKNKWWYDKGECGSGGGDSKDKTSALSLSNVAGVFYILIGGLGLAMLVALIEFCYKSRSESKRMKGFCLIPQQSINEAIRTSTLPRNSGAGASSGGSGENGRVVSHDFPKSMQSIPCMSHSSGMPLGATGL",
# GRIA2_HUMAN
"MQKIMHISVLLSPVLWGLIFGVSSNSIQIGGLFPRGADQEYSAFRVGMVQFSTSEFRLTPHIDNLEVANSFAVTNAFCSQFSRGVYAIFGFYDKKSVNTITSFCGTLHVSFITPSFPTDGTHPFVIQMRPDLKGALLSLIEYYQWDKFAYLYDSDRGLSTLQAVLDSAAEKKWQVTAINVGNINNDKKDEMYRSLFQDLELKKERRVILDCERDKVNDIVDQVITIGKHVKGYHYIIANLGFTDGDLLKIQFGGANVSGFQIVDYDDSLVSKFIERWSTLEEKEYPGAHTTTIKYTSALTYDAVQVMTEAFRNLRKQRIEISRRGNAGDCLANPAVPWGQGVEIERALKQVQVEGLSGNIKFDQNGKRINYTINIMELKTNGPRKIGYWSEVDKMVVTLTELPSGNDTSGLENKTVVVTTILESPYVMMKKNHEMLEGNERYEGYCVDLAAEIAKHCGFKYKLTIVGDGKYGARDADTKIWNGMVGELVYGKADIAIAPLTITLVREEVIDFSKPFMSLGISIMIKKPQKSKPGVFSFLDPLAYEIWMCIVFAYIGVSVVLFLVSRFSPYEWHTEEFEDGRETQSSESTNEFGIFNSLWFSLGAFMQQGCDISPRSLSGRIVGGVWWFFTLIIISSYTANLAAFLTVERMVSPIESAEDLSKQTEIAYGTLDSGSTKEFFRRSKIAVFDKMWTYMRSAEPSVFVRTTAEGVARVRKSKGKYAYLLESTMNEYIEQRKPCDTMKVGGNLDSKGYGIATPKGSSLRNAVNLAVLKLNEQGLLDKLKNKWWYDKGECGSGGGDSKEKTSALSLSNVAGVFYILVGGLGLAMLVALIEFCYKSRAEAKRMKVAKNAQNINPSSSQNSQNFATYKEGYNVYGIESVKI",
# GRIA3_HUMAN
"MARQKKMGQSVLRAVFFLVLGLLGHSHGGFPNTISIGGLFMRNTVQEHSAFRFAVQLYNTNQNTTEKPFHLNYHVDHLDSSNSFSVTNAFCSQFSRGVYAIFGFYDQMSMNTLTSFCGALHTSFVTPSFPTDADVQFVIQMRPALKGAILSLLGHYKWEKFVYLYDTERGFSILQAIMEAAVQNNWQVTARSVGNIKDVQEFRRIIEEMDRRQEKRYLIDCEVERINTILEQVVILGKHSRGYHYMLANLGFTDILLERVMHGGANITGFQIVNNENPMVQQFIQRWVRLDEREFPEAKNAPLKYTSALTHDAILVIAEAFRYLRRQRVDVSRRGSAGDCLANPAVPWSQGIDIERALKMVQVQGMTGNIQFDTYGRRTNYTIDVYEMKVSGSRKAGYWNEYERFVPFSDQQISNDSASSENRTIVVTTILESPYVMYKKNHEQLEGNERYEGYCVDLAYEIAKHVRIKYKLSIVGDGKYGARDPETKIWNGMVGELVYGRADIAVAPLTITLVREEVIDFSKPFMSLGISIMIKKPQKSKPGVFSFLDPLAYEIWMCIVFAYIGVSVVLFLVSRFSPYEWHLEDNNEEPRDPQSPPDPPNEFGIFNSLWFSLGAFMQQGCDISPRSLSGRIVGGVWWFFTLIIISSYTANLAAFLTVERMVSPIESAEDLAKQTEIAYGTLDSGSTKEFFRRSKIAVYEKMWSYMKSAEPSVFTKTTADGVARVRKSKGKFAFLLESTMNEYIEQRKPCDTMKVGGNLDSKGYGVATPKGSALRNAVNLAVLKLNEQGLLDKLKNKWWYDKGECGSGGGDSKDKTSALSLSNVAGVFYILVGGLGLAMMVALIEFCYKSRAESKRMKLTKNTQNFKPAPATNTQNYATYREGYNVYGTESVKI",
# MTOR_HUMAN
"MLGTGPAAATTAATTSSNVSVLQQFASGLKSRNEETRAKAAKELQHYVTMELREMSQEESTRFYDQLNHHIFELVSSSDANERKGGILAIASLIGVEGGNATRIGRFANYLRNLLPSNDPVVMEMASKAIGRLAMAGDTFTAEYVEFEVKRALEWLGADRNEGRRHAAVLVLRELAISVPTFFFQQVQPFFDNIFVAVWDPKQAIREGAVAALRACLILTTQREPKEMQKPQWYRHTFEEAEKGFDETLAKEKGMNRDDRIHGALLILNELVRISSMEGERLREEMEEITQQQLVHDKYCKDLMGFGTKPRHITPFTSFQAVQPQQSNALVGLLGYSSHQGLMGFGTSPSPAKSTLVESRCCRDLMEEKFDQVCQWVLKCRNSKNSLIQMTILNLLPRLAAFRPSAFTDTQYLQDTMNHVLSCVKKEKERTAAFQALGLLSVAVRSEFKVYLPRVLDIIRAALPPKDFAHKRQKAMQVDATVFTCISMLARAMGPGIQQDIKELLEPMLAVGLSPALTAVLYDLSRQIPQLKKDIQDGLLKMLSLVLMHKPLRHPGMPKGLAHQLASPGLTTLPEASDVGSITLALRTLGSFEFEGHSLTQFVRHCADHFLNSEHKEIRMEAARTCSRLLTPSIHLISGHAHVVSQTAVQVVADVLSKLLVVGITDPDPDIRYCVLASLDERFDAHLAQAENLQALFVALNDQVFEIRELAICTVGRLSSMNPAFVMPFLRKMLIQILTELEHSGIGRIKEQSARMLGHLVSNAPRLIRPYMEPILKALILKLKDPDPDPNPGVINNVLATIGELAQVSGLEMRKWVDELFIIIMDMLQDSSLLAKRQVALWTLGQLVASTGYVVEPYRKYPTLLEVLLNFLKTEQNQGTRREAIRVLGLLGALDPYKHKVNIGMIDQSRDASAVSLSESKSSQDSSDYSTSEMLVNMGNLPLDEFYPAVSMVALMRIFRDQSLSHHHTMVVQAITFIFKSLGLKCVQFLPQVMPTFLNVIRVCDGAIREFLFQQLGMLVSFVKSHIRPYMDEIVTLMREFWVMNTSIQSTIILLIEQIVVALGGEFKLYLPQLIPHMLRVFMHDNSPGRIVSIKLLAAIQLFGANLDDYLHLLLPPIVKLFDAPEAPLPSRKAALETVDRLTESLDFTDYASRIIHPIVRTLDQSPELRSTAMDTLSSLVFQLGKKYQIFIPMVNKVLVRHRINHQRYDVLICRIVKGYTLADEEEDPLIYQHRMLRSGQGDALASGPVETGPMKKLHVSTINLQKAWGAARRVSKDDWLEWLRRLSLELLKDSSSPSLRSCWALAQAYNPMARDLFNAAFVSCWSELNEDQQDELIRSIELALTSQDIAEVTQTLLNLAEFMEHSDKGPLPLRDDNGIVLLGERAAKCRAYAKALHYKELEFQKGPTPAILESLISINNKLQQPEAAAGVLEYAMKHFGELEIQATWYEKLHEWEDALVAYDKKMDTNKDDPELMLGRMRCLEALGEWGQLHQQCCEKWTLVNDETQAKMARMAAAAAWGLGQWDSMEEYTCMIPRDTHDGAFYRAVLALHQDLFSLAQQCIDKARDLLDAELTAMAGESYSRAYGAMVSCHMLSELEEVIQYKLVPERREIIRQIWWERLQGCQRIVEDWQKILMVRSLVVSPHEDMRTWLKYASLCGKSGRLALAHKTLVLLLGVDPSRQLDHPLPTVHPQVTYAYMKNMWKSARKIDAFQHMQHFVQTMQQQAQHAIATEDQQHKQELHKLMARCFLKLGEWQLNLQGINESTIPKVLQYYSAATEHDRSWYKAWHAWAVMNFEAVLHYKHQNQARDEKKKLRHASGANITNATTAATTAATATTTASTEGSNSESEAESTENSPTPSPLQKKVTEDLSKTLLMYTVPAVQGFFRSISLSRGNNLQDTLRVLTLWFDYGHWPDVNEALVEGVKAIQIDTWLQVIPQLIARIDTPRPLVGRLIHQLLTDIGRYHPQALIYPLTVASKSTTTARHNAANKILKNMCEHSNTLVQQAMMVSEELIRVAILWHEMWHEGLEEASRLYFGERNVKGMFEVLEPLHAMMERGPQTLKETSFNQAYGRDLMEAQEWCRKYMKSGNVKDLTQAWDLYYHVFRRISKQLPQLTSLELQYVSPKLLMCRDLELAVPGTYDPNQPIIRIQSIAPSLQVITSKQRPRKLTLMGSNGHEFVFLLKGHEDLRQDERVMQLFGLVNTLLANDPTSLRKNLSIQRYAVIPLSTNSGLIGWVPHCDTLHALIRDYREKKKILLNIEHRIMLRMAPDYDHLTLMQKVEVFEHAVNNTAGDDLAKLLWLKSPSSEVWFDRRTNYTRSLAVMSMVGYILGLGDRHPSNLMLDRLSGKILHIDFGDCFEVAMTREKFPEKIPFRLTRMLTNAMEVTGLDGNYRITCHTVMEVLREHKDSVMAVLEAFVYDPLLNWRLMDTNTKGNKRSRTRTDSYSAGQSVEILDGVELGEPAHKKTGTTVPESIHSFIGDGLVKPEALNKKAIQIINRVRDKLTGRDFSHDDTLDVPTQVELLIKQATSHENLCQCYIGWCPFW",
# EPS15_HUMAN
"MAAAAQLSLTQLSSGNPVYEKYYRQVDTGNTGRVLASDAAAFLKKSGLPDLILGKIWDLADTDGKGILNKQEFFVALRLVACAQNGLEVSLSSLNLAVPPPRFHDTSSPLLISGTSAAELPWAVKPEDKAKYDAIFDSLSPVNGFLSGDKVKPVLLNSKLPVDILGRVWELSDIDHDGMLDRDEFAVAMFLVYCALEKEPVPMSLPPALVPPSKRKTWVVSPAEKAKYDEIFLKTDKDMDGFVSGLEVREIFLKTGLPSTLLAHIWSLCDTKDCGKLSKDQFALAFHLISQKLIKGIDPPHVLTPEMIPPSDRASLQKNIIGSSPVADFSAIKELDTLNNEIVDLQREKNNVEQDLKEKEDTIKQRTSEVQDLQDEVQRENTNLQKLQAQKQQVQELLDELDEQKAQLEEQLKEVRKKCAEEAQLISSLKAELTSQESQISTYEEELAKAREELSRLQQETAELEESVESGKAQLEPLQQHLQDSQQEISSMQMKLMEMKDLENHNSQLNWCSSPHSILVNGATDYCSLSTSSSETANLNEHVEGQSNLESEPIHQESPARSSPELLPSGVTDENEVTTAVTEKVCSELDNNRHSKEEDPFNVDSSSLTGPVADTNLDFFQSDPFVGSDPFKDDPFGKIDPFGGDPFKGSDPFASDCFFRQSTDPFATSSTDPFSAANNSSITSVETLKHNDPFAPGGTVVAASDSATDPFASVFGNESFGGGFADFSTLSKVNNEDPFRSATSSSVSNVVITKNVFEETSVKSEDEPPALPPKIGTPTRPCPLPPGKRSINKLDSPDPFKLNDPFQPFPGNDSPKEKDPEIFCDPFTSATTTTNKEADPSNFANFSAYPSEEDMIEWAKRESEREEEQRLARLNQQEQEDLELAIALSKSEISEA",
# CASP3_HUMAN
"MENTENSVDSKSIKNLEPKIIHGSESMDSGISLDNSYKMDYPEMGLCIIINNKNFHKSTGMTSRSGTDVDAANLRETFRNLKYEVRNKNDLTREEIVELMRDVSKEDHSKRSSFVCVLLSHGEEGIIFGTNGPVDLKKITNFFRGDRCRSLTGKPKLFIIQACRGTELDCGIETDSGVDDDMACHKIPVEADFLYAYSTAPGYYSWRNSKDGSWFIQSLCAMLKQYADKLEFMHILTRVNRKVATEFESFSFDATFHAKKQIPCIVSMLTKELYFYH",
# DPP6_HUMAN
"MASLYQRFTGKINTSRSFPAPPEASHLLGGQGPEEDGGAGAKPLGPRAQAAAPRERGGGGGGAGGRPRFQYQARSDGDEEDELVGSNPPQRNWKGIAIALLVILVICSLIVTSVILLTPAEDNSLSQKKKVTVEDLFSEDFKIHDPEAKWISDTEFIYREQKGTVRLWNVETNTSTVLIEGKKIESLRAIRYEISPDREYALFSYNVEPIYQHSYTGYYVLSKIPHGDPQSLDPPEVSNAKLQYAGWGPKGQQLIFIFENNIYYCAHVGKQAIRVVSTGKEGVIYNGLSDWLYEEEILKTHIAHWWSPDGTRLAYAAINDSRVPIMELPTYTGSIYPTVKPYHYPKAGSENPSISLHVIGLNGPTHDLEMMPPDDPRMREYYITMVKWATSTKVAVTWLNRAQNVSILTLCDATTGVCTKKHEDESEAWLHRQNEEPVFSKDGRKFFFIRAIPQGGRGKFYHITVSSSQPNSSNDNIQSITSGDWDVTKILAYDEKGNKIYFLSTEDLPRRRQLYSANTVGNFNRQCLSCDLVENCTYFSASFSHSMDFFLLKCEGPGVPMVTVHNTTDKKKMFDLETNEHVKKAINDRQMPKVEYRDIEIDDYNLPMQILKPATFTDTTHYPLLLVVDGTPGSQSVAEKFEVSWETVMVSSHGAVVVKCDGRGSGFQGTKLLHEVRRRLGLLEEKDQMEAVRTMLKEQYIDRTRVAVFGKDYGGYLSTYILPAKGENQGQTFTCGSALSPITDFKLYASAFSERYLGLHGLDNRAYEMTKVAHRVSALEEQQFLIIHPTADEKIHFQHTAELITQLIRGKANYSLQIYPDESHYFTSSSLKQHLYRSIINFFVECFRIQDKLLTVTAKEDEEED",
# RS27_HUMAN
"MPLAKDLLHPSPEEEKRKHKKKRLVQSPNSYFMDVKCPGCYKITTVFSHAQTVVLCVGCSTVLCQPTGGKARLTEGCSFRRKQH",
# ABL2_HUMAN
"MGQQVGRVGEAPGLQQPQPRGIRGSSAARPSGRRRDPAGRTTETGFNIFTQHDHFASCVEDGFEGDKTGGSSPEALHRPYGCDVEPQALNEAIRWSSKENLLGATESDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNQNGEWSEVRSKNGQGWVPSNYITPVNSLEKHSWYHGPVSRSAAEYLLSSLINGSFLVRESESSPGQLSISLRYEGRVYHYRINTTADGKVYVTAESRFSTLAELVHHHSTVADGLVTTLHYPAPKCNKPTVYGVSPIHDKWEMERTDITMKHKLGGGQYGEVYVGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTLEPPFYIVTEYMPYGNLLDYLRECNREEVTAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHVVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNTFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYDLLEKGYRMEQPEGCPPKVYELMRACWKWSPADRPSFAETHQAFETMFHDSSISEEVAEELGRAASSSSVVPYLPRLPILPSKTRTLKKQVENKENIEGAQDATENSASSLAPGFIRGAQASSGSPALPRKQRDKSPSSLLEDAKETCFTRDRKGGFFSSFMKKRNAPTPPKRSSSFREMENQPHKKYELTGNFSSVASLQHADGFSFTPAQQEANLVPPKCYGGSFAQRNLCNDDGGGGGGSGTAGGGWSGITGFFTPRLIKKTLGLRAGKPTASDDTSKPFPRSNSTSSMSSGLPEQDRMAMTLPRNCQRSKLQLERTVSTSSQPEENVDRANDMLPKKSEESAAPSRERPKAKLLPRGATALPLRTPSGDLAITEKDPPGVGVAGVAAAPKGKEKNGGARLGMAGVPEDGEQPGWPSPAKAAPVLPTTHNHKVPVLISPTLKHTPADVQLIGTDSQGNKFKLLSEHQVTSSGDKDRPRRVKPKCAPPPPPVMRLLQHPSICSDPTEEPTALTAGQSTSETQEGGKKAALGAVPISGKAGRPVMPPPQVPLPTSSISPAKMANGTAGTKVALRKTKQAAEKISADKISKEALLECADLLSSALTEPVPNSQLVDTGHQLLDYCSGYVDCIPQTRNKFAFREAVSKLELSLQELQVSSAAAGVPGTNPVLNNLLSCVQEISDVVQR",
# RL35_HUMAN
"MAKIKARDLRGKKKEELLKQLDDLKVELSQLRVAKVTGGAASKLSKIRVVRKSIARVLTVINQTQKENLRKFYKGKKYKPLDLRPKKTRAMRRRLNKHEENLKTKKQQRKERLYPLRKYAVKA",
# NSG1_HUMAN
"MVKLGNNFAEKGTKQPLLEDGFDTIPLMTPLDVNQLQFPPPDKVVVKTKTEYEPDRKKGKARPPQIAEFTVSITEGVTERFKVSVLVLFALAFLTCVVFLVVYKVYKYDRACPDGFVLKNTQCIPEGLESYYAEQDSSAREKFYTVINHYNLAKQSITRSVSPWMSVLSEEKLSEQETEAAEKSA",
# HD_HUMAN
"MATLEKLMKAFESLKSFQQQQQQQQQQQQQQQQQQQQQPPPPPPPPPPPQLPQPPPQAQPLLPQPQPPPPPPPPPPGPAVAEEPLHRPKKELSATKKDRVNHCLTICENIVAQSVRNSPEFQKLLGIAMELFLLCSDDAESDVRMVADECLNKVIKALMDSNLPRLQLELYKEIKKNGAPRSLRAALWRFAELAHLVRPQKCRPYLVNLLPCLTRTSKRPEESVQETLAAAVPKIMASFGNFANDNEIKVLLKAFIANLKSSSPTIRRTAAGSAVSICQHSRRTQYFYSWLLNVLLGLLVPVEDEHSTLLILGVLLTLRYLVPLLQQQVKDTSLKGSFGVTRKEMEVSPSAEQLVQVYELTLHHTQHQDHNVVTGALELLQQLFRTPPPELLQTLTAVGGIGQLTAAKEESGGRSRSGSIVELIAGGGSSCSPVLSRKQKGKVLLGEEEALEDDSESRSDVSSSALTASVKDEISGELAASSGVSTPGSAGHDIITEQPRSQHTLQADSVDLASCDLTSSATDGDEEDILSHSSSQVSAVPSDPAMDLNDGTQASSPISDSSQTTTEGPDSAVTPSDSSEIVLDGTDNQYLGLQIGQPQDEDEEATGILPDEASEAFRNSSMALQQAHLLKNMSHCRQPSDSSVDKFVLRDEATEPGDQENKPCRIKGDIGQSTDDDSAPLVHCVRLLSASFLLTGGKNVLVPDRDVRVSVKALALSCVGAAVALHPESFFSKLYKVPLDTTEYPEEQYVSDILNYIDHGDPQVRGATAILCGTLICSILSRSRFHVGDWMGTIRTLTGNTFSLADCIPLLRKTLKDESSVTCKLACTAVRNCVMSLCSSSYSELGLQLIIDVLTLRNSSYWLVRTELLETLAEIDFRLVSFLEAKAENLHRGAHHYTGLLKLQERVLNNVVIHLLGDEDPRVRHVAAASLIRLVPKLFYKCDQGQADPVVAVARDQSSVYLKLLMHETQPPSHFSVSTITRIYRGYNLLPSITDVTMENNLSRVIAAVSHELITSTTRALTFGCCEALCLLSTAFPVCIWSLGWHCGVPPLSASDESRKSCTVGMATMILTLLSSAWFPLDLSAHQDALILAGNLLAASAPKSLRSSWASEEEANPAATKQEEVWPALGDRALVPMVEQLFSHLLKVINICAHVLDDVAPGPAIKAALPSLTNPPSLSPIRRKGKEKEPGEQASVPLSPKKGSEASAASRQSDTSGPVTTSKSSSLGSFYHLPSYLKLHDVLKATHANYKVTLDLQNSTEKFGGFLRSALDVLSQILELATLQDIGKCVEEILGYLKSCFSREPMMATVCVQQLLKTLFGTNLASQFDGLSSNPSKSQGRAQRLGSSSVRPGLYHYCFMAPYTHFTQALADASLRNMVQAEQENDTSGWFDVLQKVSTQLKTNLTSVTKNRADKNAIHNHIRLFEPLVIKALKQYTTTTCVQLQKQVLDLLAQLVQLRVNYCLLDSDQVFIGFVLKQFEYIEVGQFRESEAIIPNIFFFLVLLSYERYHSKQIIGIPKIIQLCDGIMASGRKAVTHAIPALQPIVHDLFVLRGTNKADAGKELETQKEVVVSMLLRLIQYHQVLEMFILVLQQCHKENEDKWKRLSRQIADIILPMLAKQQMHIDSHEALGVLNTLFEILAPSSLRPVDMLLRSMFVTPNTMASVSTVQLWISGILAILRVLISQSTEDIVLSRIQELSFSPYLISCTVINRLRDGDSTSTLEEHSEGKQIKNLPEETFSRFLLQLVGILLEDIVTKQLKVEMSEQQHTFYCQELGTLLMCLIHIFKSGMFRRITAAATRLFRSDGCGGSFYTLDSLNLRARSMITTHPALVLLWCQILLLVNHTDYRWWAEVQQTPKRHSLSSTKLLSPQMSGEEEDSDLAAKLGMCNREIVRRGALILFCDYVCQNLHDSEHLTWLIVNHIQDLISLSHEPPVQDFISAVHRNSAASGLFIQAIQSRCENLSTPTMLKKTLQCLEGIHLSQSGAVLTLYVDRLLCTPFRVLARMVDILACRRVEMLLAANLQSSMAQLPMEELNRIQEYLQSSGLAQRHQRLYSLLDRFRLSTMQDSLSPSPPVSSHPLDGDGHVSLETVSPDKDWYVHLVKSQCWTRSDSALLEGAELVNRIPAEDMNAFMMNSEFNLSLLAPCLSLGMSEISGGQKSALFEAAREVTLARVSGTVQQLPAVHHVFQPELPAEPAAYWSKLNDLFGDAALYQSLPTLARALAQYLVVVSKLPSHLHLPPEKEKDIVKFVVATLEALSWHLIHEQIPLSLDLQAGLDCCCLALQLPGLWSVVSSTEFVTHACSLIYCVHFILEAVAVQPGEQLLSPERRTNTPKAISEEEEEVDPNTQNPKYITAACEMVAEMVESLQSVLALGHKRNSGVPAFLTPLLRNIIISLARLPLVNSYTRVPPLVWKLGWSPKPGGDFGTAFPEIPVEFLQEKEVFKEFIYRINTLGWTSRTQFEETWATLLGVLVTQPLVMEQEESPPEEDTERTQINVLAVQAITSLVLSAMTVPVAGNPAVSCLEQQPRNKPLKALDTRFGRKLSIIRGIVEQEIQAMVSKRENIATHHLYQAWDPVPSLSPATTGALISHEKLLLQINPERELGSMSYKLGQVSIHSVWLGNSITPLREEEWDEEEEEEADAPAPSSPPTSPVNSRKHRAGVDIHSCSQFLLELYSRWILPSSSARRTPAILISEVVRSLLVVSDLFTERNQFELMYVTLTELRRVHPSEDEILAQYLVPATCKAAAVLGMDKAVAEPVSRLLESTLRSSHLPSRVGALHGVLYVLECDLLDDTAKQLIPVISDYLLSNLKGIAHCVNIHSQQHVLVMCATAFYLIENYPLDVGPEFSASIIQMCGVMLSGSEESTPSIIYHCALRGLERLLLSEQLSRLDAESLVKLSVDRVNVHSPHRAMAALGLMLTCMYTGKEKVSPGRTSDPNPAAPDSESVIVAMERVSVLFDRIRKGFPCEARVVARILPQFLDDFFPPQDIMNKVIGEFLSNQQPYPQFMATVVYKVFQTLHSTGQSSMVRDWVMLSLSNFTQRAPVAMATWSLSCFFVSASTSPWVAAILPHVISRMGKLEQVDVNLFCLVATDFYRHQIEEELDRRAFQSVLEVVAAPGSPYHRLLTCLRNVHKVTTC",
# EAA2_HUMAN
"MASTEGANNMPKQVEVRMHDSHLGSEEPKHRHLGLRLCDKLGKNLLLTLTVFGVILGAVCGGLLRLASPIHPDVVMLIAFPGDILMRMLKMLILPLIISSLITGLSGLDAKASGRLGTRAMVYYMSTTIIAAVLGVILVLAIHPGNPKLKKQLGPGKKNDEVSSLDAFLDLIRNLFPENLVQACFQQIQTVTKKVLVAPPPDEEANATSAVVSLLNETVTEVPEETKMVIKKGLEFKDGMNVLGLIGFFIAFGIAMGKMGDQAKLMVDFFNILNEIVMKLVIMIMWYSPLGIACLICGKIIAIKDLEVVARQLGMYMVTVIIGLIIHGGIFLPLIYFVVTRKNPFSFFAGIFQAWITALGTASSAGTLPVTFRCLEENLGIDKRVTRFVLPVGATINMDGTALYEAVAAIFIAQMNGVVLDGGQIVTVSLTATLASVGAASIPSAGLVTMLLILTAVGLPTEDISLLVAVDWLLDRMRTSVNVVGDSFGAGIVYHLSKSELDTIDSQHRVHEDIEMTKTQSIYDDMKNHRESNSNQCVYAAHNSVIVDECKVTLAANGKSADCSVEEEPWKREK",
# EAA3_HUMAN
"MGKPARKGCEWKRFLKNNWVLLSTVAAVVLGITTGVLVREHSNLSTLEKFYFAFPGEILMRMLKLIILPLIISSMITGVAALDSNVSGKIGLRAVVYYFCTTLIAVILGIVLVVSIKPGVTQKVGEIARTGSTPEVSTVDAMLDLIRNMFPENLVQACFQQYKTKREEVKPPSDPEMNMTEESFTAVMTTAISKNKTKEYKIVGMYSDGINVLGLIVFCLVFGLVIGKMGEKGQILVDFFNALSDATMKIVQIIMCYMPLGILFLIAGKIIEVEDWEIFRKLGLYMATVLTGLAIHSIVILPLIYFIVVRKNPFRFAMGMAQALLTALMISSSSATLPVTFRCAEENNQVDKRITRFVLPVGATINMDGTALYEAVAAVFIAQLNDLDLGIGQIITISITATSASIGAAGVPQAGLVTMVIVLSAVGLPAEDVTLIIAVDWLLDRFRTMVNVLGDAFGTGIVEKLSKKELEQMDVSSEVNIVNPFALESTILDNEDSDTKKSYVNGGFAVDKSDTISFTQTSQF",
# LIS1_HUMAN
"MVLSQRQRDELNRAIADYLRSNGYEEAYSVFKKEAELDVNEELDKKYAGLLEKKWTSVIRLQKKVMELESKLNEAKEEFTSGGPLGQKRDPKEWIPRPPEKYALSGHRSPVTRVIFHPVFSVMVSASEDATIKVWDYETGDFERTLKGHTDSVQDISFDHSGKLLASCSADMTIKLWDFQGFECIRTMHGHDHNVSSVAIMPNGDHIVSASRDKTIKMWEVQTGYCVKTFTGHREWVRMVRPNQDGTLIASCSNDQTVRVWVVATKECKAELREHEHVVECISWAPESSYSSISEATGSETKKSGKPGPFLLSGSRDKTIKMWDVSTGMCLMTLVGHDNWVRGVLFHSGGKFILSCADDKTLRVWDYKNKRCMKTLNAHEHFVTSLDFHKTAPYVVTGSVDQTVKVWECR",
# DCC_HUMAN
"MENSLRCVWVPKLAFVLFGASLFSAHLQVTGFQIKAFTALRFLSEPSDAVTMRGGNVLLDCSAESDRGVPVIKWKKDGIHLALGMDERKQQLSNGSLLIQNILHSRHHKPDEGLYQCEASLGDSGSIISRTAKVAVAGPLRFLSQTESVTAFMGDTVLLKCEVIGEPMPTIHWQKNQQDLTPIPGDSRVVVLPSGALQISRLQPGDIGIYRCSARNPASSRTGNEAEVRILSDPGLHRQLYFLQRPSNVVAIEGKDAVLECCVSGYPPPSFTWLRGEEVIQLRSKKYSLLGGSNLLISNVTDDDSGMYTCVVTYKNENISASAELTVLVPPWFLNHPSNLYAYESMDIEFECTVSGKPVPTVNWMKNGDVVIPSDYFQIVGGSNLRILGVVKSDEGFYQCVAENEAGNAQTSAQLIVPKPAIPSSSVLPSAPRDVVPVLVSSRFVRLSWRPPAEAKGNIQTFTVFFSREGDNRERALNTTQPGSLQLTVGNLKPEAMYTFRVVAYNEWGPGESSQPIKVATQPELQVPGPVENLQAVSTSPTSILITWEPPAYANGPVQGYRLFCTEVSTGKEQNIEVDGLSYKLEGLKKFTEYSLRFLAYNRYGPGVSTDDITVVTLSDVPSAPPQNVSLEVVNSRSIKVSWLPPPSGTQNGFITGYKIRHRKTTRRGEMETLEPNNLWYLFTGLEKGSQYSFQVSAMTVNGTGPPSNWYTAETPENDLDESQVPDQPSSLHVRPQTNCIIMSWTPPLNPNIVVRGYIIGYGVGSPYAETVRVDSKQRYYSIERLESSSHYVISLKAFNNAGEGVPLYESATTRSITDPTDPVDYYPLLDDFPTSVPDLSTPMLPPVGVQAVALTHDAVRVSWADNSVPKNQKTSEVRLYTVRWRTSFSASAKYKSEDTTSLSYTATGLKPNTMYEFSVMVTKNRRSSTWSMTAHATTYEAAPTSAPKDLTVITREGKPRAVIVSWQPPLEANGKITAYILFYTLDKNIPIDDWIMETISGDRLTHQIMDLNLDTMYYFRIQARNSKGVGPLSDPILFRTLKVEHPDKMANDQGRHGDGGYWPVDTNLIDRSTLNEPPIGQMHPPHGSVTPQKNSNLLVIIVVTVGVITVLVVVIVAVICTRRSSAQQRKKRATHSAGKRKGSQKDLRPPDLWIHHEEMEMKNIEKPSGTDPAGRDSPIQSCQDLTPVSHSQSETQLGSKSTSHSGQDTEEAGSSMSTLERSLAARRAPRAKLMIPMDAQSNNPAVVSAIPVPTLESAQYPGILPSPTCGYPHPQFTLRPVPFPTLSVDRGFGAGRSQSVSEGPTTQQPPMLPPSQPEHSSSEEAPSRTIPTACVRPTHPLRSFANPLLPPPMSAIEPKVPYTPLLSQPGPTLPKTHVKTASLGLAGKARSPLLPVSVPTAPEVSEESHKPTEDSANVYEQDDLSEQMASLEGLMKQLNAITGSAF",
# ACHA4_HUMAN
"MELGGPGAPRLLPPLLLLLGTGLLRASSHVETRAHAEERLLKKLFSGYNKWSRPVANISDVVLVRFGLSIAQLIDVDEKNQMMTTNVWVKQEWHDYKLRWDPADYENVTSIRIPSELIWRPDIVLYNNADGDFAVTHLTKAHLFHDGRVQWTPPAIYKSSCSIDVTFFPFDQQNCTMKFGSWTYDKAKIDLVNMHSRVDQLDFWESGEWVIVDAVGTYNTRKYECCAEIYPDITYAFVIRRLPLFYTINLIIPCLLISCLTVLVFYLPSECGEKITLCISVLLSLTVFLLLITEIIPSTSLVIPLIGEYLLFTMIFVTLSIVITVFVLNVHHRSPRTHTMPTWVRRVFLDIVPRLLLMKRPSVVKDNCRRLIESMHKMASAPRFWPEPEGEPPATSGTQSLHPPSPSFCVPLDVPAEPGPSCKSPSDQLPPQQPLEAEKASPHPSPGPCRPPHGTQAPGLAKARSLSVQHMSSPGEAVEGGVRCRSRSIQYCVPRDDAAPEADGQAAGALASRNTHSAELPPPDQPSPCKCTCKKEPSSVSPSATVKTRSTKAPPPHLPLSPALTRAVEGVQYIADHLKAEDTDFSVKEDWKYVAMVIDRIFLWMFIIVCLLGTVGLFLPPWLAGMI",
# UBP5_HUMAN
"MAELSEEALLSVLPTIRVPKAGDRVHKDECAFSFDTPESEGGLYICMNTFLGFGKQYVERHFNKTGQRVYLHLRRTRRPKEEDPATGTGDPPRKKPTRLAIGVEGGFDLSEEKFELDEDVKIVILPDYLEIARDGLGGLPDIVRDRVTSAVEALLSADSASRKQEVQAWDGEVRQVSKHAFSLKQLDNPARIPPCGWKCSKCDMRENLWLNLTDGSILCGRRYFDGSGGNNHAVEHYRETGYPLAVKLGTITPDGADVYSYDEDDMVLDPSLAEHLSHFGIDMLKMQKTDKTMTELEIDMNQRIGEWELIQESGVPLKPLFGPGYTGIRNLGNSCYLNSVVQVLFSIPDFQRKYVDKLEKIFQNAPTDPTQDFSTQVAKLGHGLLSGEYSKPVPESGDGERVPEQKEVQDGIAPRMFKALIGKGHPEFSTNRQQDAQEFFLHLINMVERNCRSSENPNEVFRFLVEEKIKCLATEKVKYTQRVDYIMQLPVPMDAALNKEELLEYEEKKRQAEEEKMALPELVRAQVPFSSCLEAYGAPEQVDDFWSTALQAKSVAVKTTRFASFPDYLVIQIKKFTFGLDWVPKKLDVSIEMPEELDISQLRGTGLQPGEEELPDIAPPLVTPDEPKGSLGFYGNEDEDSFCSPHFSSPTSPMLDESVIIQLVEMGFPMDACRKAVYYTGNSGAEAAMNWVMSHMDDPDFANPLILPGSSGPGSTSAAADPPPEDCVTTIVSMGFSRDQALKALRATNNSLERAVDWIFSHIDDLDAEAAMDISEGRSAADSISESVPVGPKVRDGPGKYQLFAFISHMGTSTMCGHYVCHIKKEGRWVIYNDQKVCASEKPPKDLGYIYFYQRVAS",
# MK08_HUMAN
"MSRSKRDNNFYSVEIGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDAILERNVAIKKLSRPFQNQTHAKRAYRELVLMKCVNHKNIIGLLNVFTPQKSLEEFQDVYIVMELMDANLCQVIQMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTAGTSFMMTPYVVTRYYRAPEVILGMGYKENVDLWSVGCIMGEMVCHKILFPGRDYIDQWNKVIEQLGTPCPEFMKKLQPTVRTYVENRPKYAGYSFEKLFPDVLFPADSEHNKLKASQARDLLSKMLVIDASKRISVDEALQHPYINVWYDPSEAEAPPPKIPDKQLDEREHTIEEWKELIYKEVMDLEERTKNGVIRGQPSPLGAAVINGSQHPSSSSSVNDVSSMSTDPTLASDTDSSLEAAAGPLGCCR",
# MK09_HUMAN
"MSDSKCDSQFYSVQVADSTFTVLKRYQQLKPIGSGAQGIVCAAFDTVLGINVAVKKLSRPFQNQTHAKRAYRELVLLKCVNHKNIISLLNVFTPQKTLEEFQDVYLVMELMDANLCQVIHMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTACTNFMMTPYVVTRYYRAPEVILGMGYKENVDIWSVGCIMGELVKGCVIFQGTDHIDQWNKVIEQLGTPSAEFMKKLQPTVRNYVENRPKYPGIKFEELFPDWIFPSESERDKIKTSQARDLLSKMLVIDPDKRISVDEALRHPYITVWYDPAEAEAPPPQIYDAQLEEREHAIEEWKELIYKEVMDWEERSKNGVVKDQPSDAAVSSNATPSQSSSINDISSMSTEQTLASDTDSSLDASTGPLEGCR",
# 5HT3A_HUMAN
"MLLWVQQALLALLLPTLLAQGEARRSRNTTRPALLRLSDYLLTNYRKGVRPVRDWRKPTTVSIDVIVYAILNVDEKNQVLTTYIWYRQYWTDEFLQWNPEDFDNITKLSIPTDSIWVPDILINEFVDVGKSPNIPYVYIRHQGEVQNYKPLQVVTACSLDIYNFPFDVQNCSLTFTSWLHTIQDINISLWRLPEKVKSDRSVFMNQGEWELLGVLPYFREFSMESSNYYAEMKFYVVIRRRPLFYVVSLLLPSIFLMVMDIVGFYLPPNSGERVSFKITLLLGYSVFLIIVSDTLPATAIGTPLIGVYFVVCMALLVISLAETIFIVRLVHKQDLQQPVPAWLRHLVLERIAWLLCLREQSTSQRPPATSQATKTDDCSAMGNHCSHMGGPQDFEKSPRDRCSPPPPPREASLAVCGLLQELSSIRQFLEKRDEIREVARDWLRVGSVLDKLLFHIYLLAVLAYSITLVMLWSIWQYA",
# CRK_HUMAN
"MAGNFDSEERSSWYWGRLSRQEAVALLQGQRHGVFLVRDSSTSPGDYVLSVSENSRVSHYIINSSGPRPPVPPSPAQPPPGVSPSRLRIGDQEFDSLPALLEFYKIHYLDTTTLIEPVSRSRQGSGVILRQEEAEYVRALFDFNGNDEEDLPFKKGDILRIRDKPEEQWWNAEDSEGKRGMIPVPYVEKYRPASASVSALIGGNQEGSHPQPLGGPEPGPYAQPSVNTPLPNLQNGPIYARVIQKRVPNAYDKTALALEVGELVKVTKINVSGQWEGECNGKRGHFPFTHVRLLDQQNPDEDFS",
# CRKL_HUMAN
"MSSARFDSSDRSAWYMGPVSRQEAQTRLQGQRHGMFLVRDSSTCPGDYVLSVSENSRVSHYIINSLPNRRFKIGDQEFDHLPALLEFYKIHYLDTTTLIEPAPRYPSPPMGSVSAPNLPTAEDNLEYVRTLYDFPGNDAEDLPFKKGEILVIIEKPEEQWWSARNKDGRVGMIPVPYVEKLVRSSPHGKHGNRNSNSYGIPEPAHAYAQPQTTTPLPAVSGSPGAAITPLPSTQNGPVFAKAIQKRVPCAYDKTALALEVGDIVKVTRMNINGQWEGEVNGRKGLFPFTHVKIFDPQNPDENE",
# NSF_HUMAN
"MAGRSMQAARCPTDELSLTNCAVVNEKDFQSGQHVIVRTSPNHRYTFTLKTHPSVVPGSIAFSLPQRKWAGLSIGQEIEVSLYTFDKAKQCIGTMTIEIDFLQKKSIDSNPYDTDKMAAEFIQQFNNQAFSVGQQLVFSFNEKLFGLLVKDIEAMDPSILKGEPATGKRQKIEVGLVVGNSQVAFEKAENSSLNLIGKAKTKENRQSIINPDWNFEKMGIGGLDKEFSDIFRRAFASRVFPPEIVEQMGCKHVKGILLYGPPGCGKTLLARQIGKMLNAREPKVVNGPEILNKYVGESEANIRKLFADAEEEQRRLGANSGLHIIIFDEIDAICKQRGSMAGSTGVHDTVVNQLLSKIDGVEQLNNILVIGMTNRPDLIDEALLRPGRLEVKMEIGLPDEKGRLQILHIHTARMRGHQLLSADVDIKELAVETKNFSGAELEGLVRAAQSTAMNRHIKASTKVEVDMEKAESLQVTRGDFLASLENDIKPAFGTNQEDYASYIMNGIIKWGDPVTRVLDDGELLVQQTKNSDRTPLVSVLLEGPPHSGKTALAAKIAEESNFPFIKICSPDKMIGFSETAKCQAMKKIFDDAYKSQLSCVVVDDIERLLDYVPIGPRFSNLVLQALLVLLKKAPPQGRKLLIIGTTSRKDVLQEMEMLNAFSTTIHVPNIATGEQLLEALELLGNFKDKERTTIAQQVKGKKVWIGIKKLLMLIEMSLQMDPEYRVRKFLALLREEGASPLDFD",
# NOTC1_HUMAN
"MPPLLAPLLCLALLPALAARGPRCSQPGETCLNGGKCEAANGTEACVCGGAFVGPRCQDPNPCLSTPCKNAGTCHVVDRRGVADYACSCALGFSGPLCLTPLDNACLTNPCRNGGTCDLLTLTEYKCRCPPGWSGKSCQQADPCASNPCANGGQCLPFEASYICHCPPSFHGPTCRQDVNECGQKPGLCRHGGTCHNEVGSYRCVCRATHTGPNCERPYVPCSPSPCQNGGTCRPTGDVTHECACLPGFTGQNCEENIDDCPGNNCKNGGACVDGVNTYNCRCPPEWTGQYCTEDVDECQLMPNACQNGGTCHNTHGGYNCVCVNGWTGEDCSENIDDCASAACFHGATCHDRVASFYCECPHGRTGLLCHLNDACISNPCNEGSNCDTNPVNGKAICTCPSGYTGPACSQDVDECSLGANPCEHAGKCINTLGSFECQCLQGYTGPRCEIDVNECVSNPCQNDATCLDQIGEFQCICMPGYEGVHCEVNTDECASSPCLHNGRCLDKINEFQCECPTGFTGHLCQYDVDECASTPCKNGAKCLDGPNTYTCVCTEGYTGTHCEVDIDECDPDPCHYGSCKDGVATFTCLCRPGYTGHHCETNINECSSQPCRHGGTCQDRDNAYLCFCLKGTTGPNCEINLDDCASSPCDSGTCLDKIDGYECACEPGYTGSMCNINIDECAGNPCHNGGTCEDGINGFTCRCPEGYHDPTCLSEVNECNSNPCVHGACRDSLNGYKCDCDPGWSGTNCDINNNECESNPCVNGGTCKDMTSGYVCTCREGFSGPNCQTNINECASNPCLNQGTCIDDVAGYKCNCLLPYTGATCEVVLAPCAPSPCRNGGECRQSEDYESFSCVCPTGWQGQTCEVDINECVLSPCRHGASCQNTHGGYRCHCQAGYSGRNCETDIDDCRPNPCHNGGSCTDGINTAFCDCLPGFRGTFCEEDINECASDPCRNGANCTDCVDSYTCTCPAGFSGIHCENNTPDCTESSCFNGGTCVDGINSFTCLCPPGFTGSYCQHDVNECDSQPCLHGGTCQDGCGSYRCTCPQGYTGPNCQNLVHWCDSSPCKNGGKCWQTHTQYRCECPSGWTGLYCDVPSVSCEVAAQRQGVDVARLCQHGGLCVDAGNTHHCRCQAGYTGSYCEDLVDECSPSPCQNGATCTDYLGGYSCKCVAGYHGVNCSEEIDECLSHPCQNGGTCLDLPNTYKCSCPRGTQGVHCEINVDDCNPPVDPVSRSPKCFNNGTCVDQVGGYSCTCPPGFVGERCEGDVNECLSNPCDARGTQNCVQRVNDFHCECRAGHTGRRCESVINGCKGKPCKNGGTCAVASNTARGFICKCPAGFEGATCENDARTCGSLRCLNGGTCISGPRSPTCLCLGPFTGPECQFPASSPCLGGNPCYNQGTCEPTSESPFYRCLCPAKFNGLLCHILDYSFGGGAGRDIPPPLIEEACELPECQEDAGNKVCSLQCNNHACGWDGGDCSLNFNDPWKNCTQSLQCWKYFSDGHCDSQCNSAGCLFDGFDCQRAEGQCNPLYDQYCKDHFSDGHCDQGCNSAECEWDGLDCAEHVPERLAAGTLVVVVLMPPEQLRNSSFHFLRELSRVLHTNVVFKRDAHGQQMIFPYYGREEELRKHPIKRAAEGWAAPDALLGQVKASLLPGGSEGGRRRRELDPMDVRGSIVYLEIDNRQCVQASSQCFQSATDVAAFLGALASLGSLNIPYKIEAVQSETVEPPPPAQLHFMYVAAAAFVLLFFVGCGVLLSRKRRRQHGQLWFPEGFKVSEASKKKRREPLGEDSVGLKPLKNASDGALMDDNQNEWGDEDLETKKFRFEEPVVLPDLDDQTDHRQWTQQHLDAADLRMSAMAPTPPQGEVDADCMDVNVRGPDGFTPLMIASCSGGGLETGNSEEEEDAPAVISDFIYQGASLHNQTDRTGETALHLAARYSRSDAAKRLLEASADANIQDNMGRTPLHAAVSADAQGVFQILIRNRATDLDARMHDGTTPLILAARLAVEGMLEDLINSHADVNAVDDLGKSALHWAAAVNNVDAAVVLLKNGANKDMQNNREETPLFLAAREGSYETAKVLLDHFANRDITDHMDRLPRDIAQERMHHDIVRLLDEYNLVRSPQLHGAPLGGTPTLSPPLCSPNGYLGSLKPGVQGKKVRKPSSKGLACGSKEAKDLKARRKKSQDGKGCLLDSSGMLSPVDSLESPHGYLSDVASPPLLPSPFQQSPSVPLNHLPGMPDTHLGIGHLNVAAKPEMAALGGGGRLAFETGPPRLSHLPVASGTSTVLGSSSGGALNFTVGGSTSLNGQCEWLSRLQSGMVPNQYNPLRGSVAPGPLSTQAPSLQHGMVGPLHSSLAASALSQMMSYQGLPSTRLATQPHLVQTQQVQPQNLQMQQQNLQPANIQQQQSLQPPPPPPQPHLGVSSAASGHLGRSFLSGEPSQADVQPLGPSSLAVHTILPQESPALPTSLPSSLVPPVTAAQFLTPPSQHSYSSPVDNTPSHQLQVPEHPFLTPSPESPDQWSSSSPHSNVSDWSEGVSSPPTSMQSQIARIPEAFK",
# RL27A_HUMAN
"MPSRLRKTRKLRGHVSHGHGRIGKHRKHPGGRGNAGGLHHHRINFDKYHPGYFGKVGMKHYHLKRNQSFCPTVNLDKLWTLVSEQTRVNAAKNKTGAAPIIDVVRSGYYKVLGKGKLPKQPVIVKAKFFSRRAEEKIKSVGGACVLVA",
# RL5_HUMAN
"MGFVKVVKNKAYFKRYQVKFRRRREGKTDYYARKRLVIQDKNKYNTPKYRMIVRVTNRDIICQIAYARIEGDMIVCAAYAHELPKYGVKVGLTNYAAAYCTGLLLARRLLNRFGMDKIYEGQVEVTGDEYNVESIDGQPGAFTCYLDAGLARTTTGNKVFGALKGAVDGGLSIPHSTKRFPGYDSESKEFNAEVHRKHIMGQNVADYMRYLMEEDEDAYKKQFSQYIKNSVTPDMMEEMYKKAHAAIRENPVYEKKPKKEVKKKRWNRPKMSLAQKKDRVAQKKASFLRAQERAAES",
# RL21_HUMAN
"MTNTKGKRRGTRYMFSRPFRKHGVVPLATYMRIYKKGDIVDIKGMGTVQKGMPHKCYHGKTGRVYNVTQHAVGIVVNKQVKGKILAKRINVRIEHIKHSKSRDSFLKRVKENDQKKKEAKEKGTWVQLKRQPAPPREAHFVRTNGKEPELLEPIPYEFMA",
# RL28_HUMAN
"MSAHLQWMVVRNCSSFLIKRNKQTYSTEPNNLKARNSFRYNGLIHRKTVGVEPAADGKGVVVVIKRRSGQRKPATSYVRTTINKNARATLSSIRHMIRKNKYRPDLRMAAIRRASAILRSQKPVMVKRKRTRPTKSS",
# RS9_HUMAN
"MPVARSWVCRKTYVTPRRPFEKSRLDQELKLIGEYGLRNKREVWRVKFTLAKIRKAARELLTLDEKDPRRLFEGNALLRRLVRIGVLDEGKMKLDYILGLKIEDFLERRLQTQVFKLGLAKSIHHARVLIRQRHIRVRKQVVNIPSFIVRLDSQKHIDFSLRSPYGGGRPGRVKRKNAKKGQGGAGAGDDEEED",
# RS5_HUMAN
"MTEWETAAPAVAETPDIKLFGKWSTDDVQINDISLQDYIAVKEKYAKYLPHSAGRYAAKRFRKAQCPIVERLTNSMMMHGRNNGKKLMTVRIVKHAFEIIHLLTGENPLQVLVNAIINSGPREDSTRIGRAGTVRRQAVDVSPLRRVNQAIWLLCTGAREAAFRNIKTIAECLADELINAAKGSSNSYAIKKKDELERVAKSNR",
# RS10_HUMAN
"MLMPKKNRIAIYELLFKEGVMVAKKDVHMPKHPELADKNVPNLHVMKAMQSLKSRGYVKEQFAWRHFYWYLTNEGIQYLRDYLHLPPEIVPATLRRSRPETGRPRPKGLEGERPARLTRGEADRDTYRRSAVPPGADKKAEAGAGSATEFQFRGGFGRGRGQPPQ",
# MAP1B_HUMAN
"MATVVVEATEPEPSGSIANPAASTSPSLSHRFLDSKFYLLVVVGEIVTEEHLRRAIGNIELGIRSWDTNLIECNLDQELKLFVSRHSARFSPEVPGQKILHHRSDVLETVVLINPSDEAVSTEVRLMITDAARHKLLVLTGQCFENTGELILQSGSFSFQNFIEIFTDQEIGELLSTTHPANKASLTLFCPEEGDWKNSNLDRHNLQDFINIKLNSASILPEMEGLSEFTEYLSESVEVPSPFDILEPPTSGGFLKLSKPCCYIFPGGRGDSALFAVNGFNMLINGGSERKSCFWKLIRHLDRVDSILLTHIGDDNLPGINSMLQRKIAELEEEQSQGSTTNSDWMKNLISPDLGVVFLNVPENLKNPEPNIKMKRSIEEACFTLQYLNKLSMKPEPLFRSVGNTIDPVILFQKMGVGKLEMYVLNPVKSSKEMQYFMQQWTGTNKDKAEFILPNGQEVDLPISYLTSVSSLIVWHPANPAEKIIRVLFPGNSTQYNILEGLEKLKHLDFLKQPLATQKDLTGQVPTPVVKQTKLKQRADSRESLKPAAKPLPSKSVRKESKEETPEVTKVNHVEKPPKVESKEKVMVKKDKPIKTETKPSVTEKEVPSKEEPSPVKAEVAEKQATDVKPKAAKEKTVKKETKVKPEDKKEEKEKPKKEVAKKEDKTPIKKEEKPKKEEVKKEVKKEIKKEEKKEPKKEVKKETPPKEVKKEVKKEEKKEVKKEEKEPKKEIKKLPKDAKKSSTPLSEAKKPAALKPKVPKKEESVKKDSVAAGKPKEKGKIKVIKKEGKAAEAVAAAVGTGATTAAVMAAAGIAAIGPAKELEAERSLMSSPEDLTKDFEELKAEEVDVTKDIKPQLELIEDEEKLKETEPVEAYVIQKEREVTKGPAESPDEGITTTEGEGECEQTPEELEPVEKQGVDDIEKFEDEGAGFEESSETGDYEEKAETEEAEEPEEDGEEHVCVSASKHSPTEDEESAKAEADAYIREKRESVASGDDRAEEDMDEAIEKGEAEQSEEEADEEDKAEDAREEEYEPEKMEAEDYVMAVVDKAAEAGGAEEQYGFLTTPTKQLGAQSPGREPASSIHDETLPGGSESEATASDEENREDQPEEFTATSGYTQSTIEISSEPTPMDEMSTPRDVMSDETNNEETESPSQEFVNITKYESSLYSQEYSKPADVTPLNGFSEGSKTDATDGKDYNASASTISPPSSMEEDKFSRSALRDAYCSEVKASTTLDIKDSISAVSSEKVSPSKSPSLSPSPPSPLEKTPLGERSVNFSLTPNEIKVSAEAEVAPVSPEVTQEVVEEHCASPEDKTLEVVSPSQSVTGSAGHTPYYQSPTDEKSSHLPTEVIEKPPAVPVSFEFSDAKDENERASVSPMDEPVPDSESPIEKVLSPLRSPPLIGSESAYESFLSADDKASGRGAESPFEEKSGKQGSPDQVSPVSEMTSTSLYQDKQEGKSTDFAPIKEDFGQEKKTDDVEAMSSQPALALDERKLGDVSPTQIDVSQFGSFKEDTKMSISEGTVSDKSATPVDEGVAEDTYSHMEGVASVSTASVATSSFPEPTTDDVSPSLHAEVGSPHSTEVDDSLSVSVVQTPTTFQETEMSPSKEECPRPMSISPPDFSPKTAKSRTPVQDHRSEQSSMSIEFGQESPEQSLAMDFSRQSPDHPTVGAGVLHITENGPTEVDYSPSDMQDSSLSHKIPPMEEPSYTQDNDLSELISVSQVEASPSTSSAHTPSQIASPLQEDTLSDVAPPRDMSLYASLTSEKVQSLEGEKLSPKSDISPLTPRESSPLYSPTFSDSTSAVKEKTATCHSSSSPPIDAASAEPYGFRASVLFDTMQHHLALNRDLSTPGLEKDSGGKTPGDFSYAYQKPEETTRSPDEEDYDYESYEKTTRTSDVGGYYYEKIERTTKSPSDSGYSYETIGKTTKTPEDGDYSYEIIEKTTRTPEEGGYSYDISEKTTSPPEVSGYSYEKTERSRRLLDDISNGYDDSEDGGHTLGDPSYSYETTEKITSFPESEGYSYETSTKTTRTPDTSTYCYETAEKITRTPQASTYSYETSDLCYTAEKKSPSEARQDVDLCLVSSCEYKHPKTELSPSFINPNPLEWFASEEPTEESEKPLTQSGGAPPPPGGKQQGRQCDETPPTSVSESAPSQTDSDVPPETEECPSITADANIDSEDESETIPTDKTVTYKHMDPPPAPVQDRSPSPRHPDVSMVDPEALAIEQNLGKALKKDLKEKTKTKKPGTKTKSSSPVKKSDGKSKPLAASPKPAGLKESSDKVSRVASPKKKESVEKAAKPTTTPEVKAARGEEKDKETKNAANASASKSAKTATAGPGTTKTTKSSAVPPGLPVYLDLCYIPNHSNSKNVDVEFFKRVRSSYYVVSGNDPAAEEPSRAVLDALLEGKAQWGSNMQVTLIPTHDSEVMREWYQETHEKQQDLNIMVLASSSTVVMQDESFPACKIEL",
# NEDD4_HUMAN
"MAQSLRLHFAARRSNTYPLSETSGDDLDSHVHMCFKRPTRISTSNVVQMKLTPRQTALAPLIKENVQSQERSSVPSSENVNKKSSCLQISLQPTRYSGYLQSSNVLADSDDASFTCILKDGIYSSAVVDNELNAVNDGHLVSSPAICSGSLSNFSTSDNGSYSSNGSDFGSCASITSGGSYTNSVISDSSSYTFPPSDDTFLGGNLPSDSTSNRSVPNRNTTPCEIFSRSTSTDPFVQDDLEHGLEIMKLPVSRNTKIPLKRYSSLVIFPRSPSTTRPTSPTSLCTLLSKGSYQTSHQFIISPSEIAHNEDGTSAKGFLSTAVNGLRLSKTICTPGEVRDIRPLHRKGSLQKKIVLSNNTPRQTVCEKSSEGYSCVSVHFTQRKAATLDCETTNGDCKPEMSEIKLNSDSEYIKLMHRTSACLPSSQNVDCQININGELERPHSQMNKNHGILRRSISLGGAYPNISCLSSLKHNCSKGGPSQLLIKFASGNEGKVDNLSRDSNRDCTNELSNSCKTRDDFLGQVDVPLYPLPTENPRLERPYTFKDFVLHPRSHKSRVKGYLRLKMTYLPKTSGSEDDNAEQAEELEPGWVVLDQPDAACHLQQQQEPSPLPPGWEERQDILGRTYYVNHESRRTQWKRPTPQDNLTDAENGNIQLQAQRAFTTRRQISEETESVDNRESSENWEIIREDEATMYSNQAFPSPPPSSNLDVPTHLAEELNARLTIFGNSAVSQPASSSNHSSRRGSLQAYTFEEQPTLPVLLPTSSGLPPGWEEKQDERGRSYYVDHNSRTTTWTKPTVQATVETSQLTSSQSSAGPQSQASTSDSGQQVTQPSEIEQGFLPKGWEVRHAPNGRPFFIDHNTKTTTWEDPRLKIPAHLRGKTSLDTSNDLGPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRLENVAITGPAVPYSRDYKRKYEFFRRKLKKQNDIPNKFEMKLRRATVLEDSYRRIMGVKRADFLKARLWIEFDGEKGLDYGGVAREWFFLISKEMFNPYYGLFEYSATDNYTLQINPNSGLCNEDHLSYFKFIGRVAGMAVYHGKLLDGFFIRPFYKMMLHKPITLHDMESVDSEYYNSLRWILENDPTELDLRFIIDEELFGQTHQHELKNGGSEIVVTNKNKKEYIYLVIQWRFVNRIQKQMAAFKEGFFELIPQDLIKIFDENELELLMCGLGDVDVNDWREHTKYKNGYSANHQVIQWFWKAVLMMDSEKRIRLLQFVTGTSRVPMNGFAELYGSNGPQSFTVEQWGTPEKLPRAHTCFNRLDLPPYESFEELWDKLQMAIENTQGFDGVD",
# IQGA1_HUMAN
"MSAADEVDGLGVARPHYGSVLDNERLTAEEMDERRRQNVAYEYLCHLEEAKRWMEACLGEDLPPTTELEEGLRNGVYLAKLGNFFSPKVVSLKKIYDREQTRYKATGLHFRHTDNVIQWLNAMDEIGLPKIFYPETTDIYDRKNMPRCIYCIHALSLYLFKLGLAPQIQDLYGKVDFTEEEINNMKTELEKYGIQMPAFSKIGGILANELSVDEAALHAAVIAINEAIDRRIPADTFAALKNPNAMLVNLEEPLASTYQDILYQAKQDKMTNAKNRTENSERERDVYEELLTQAEIQGNINKVNTFSALANIDLALEQGDALALFRALQSPALGLRGLQQQNSDWYLKQLLSDKQQKRQSGQTDPLQKEELQSGVDAANSAAQQYQRRLAAVALINAAIQKGVAEKTVLELMNPEAQLPQVYPFAADLYQKELATLQRQSPEHNLTHPELSVAVEMLSSVALINRALESGDVNTVWKQLSSSVTGLTNIEEENCQRYLDELMKLKAQAHAENNEFITWNDIQACVDHVNLVVQEEHERILAIGLINEALDEGDAQKTLQALQIPAAKLEGVLAEVAQHYQDTLIRAKREKAQEIQDESAVLWLDEIQGGIWQSNKDTQEAQKFALGIFAINEAVESGDVGKTLSALRSPDVGLYGVIPECGETYHSDLAEAKKKKLAVGDNNSKWVKHWVKGGYYYYHNLETQEGGWDEPPNFVQNSMQLSREEIQSSISGVTAAYNREQLWLANEGLITRLQARCRGYLVRQEFRSRMNFLKKQIPAITCIQSQWRGYKQKKAYQDRLAYLRSHKDEVVKIQSLARMHQARKRYRDRLQYFRDHINDIIKIQAFIRANKARDDYKTLINAEDPPMVVVRKFVHLLDQSDQDFQEELDLMKMREEVITLIRSNQQLENDLNLMDIKIGLLVKNKITLQDVVSHSKKLTKKNKEQLSDMMMINKQKGGLKALSKEKREKLEAYQHLFYLLQTNPTYLAKLIFQMPQNKSTKFMDSVIFTLYNYASNQREEYLLLRLFKTALQEEIKSKVDQIQEIVTGNPTVIKMVVSFNRGARGQNALRQILAPVVKEIMDDKSLNIKTDPVDIYKSWVNQMESQTGEASKLPYDVTPEQALAHEEVKTRLDSSIRNMRAVTDKFLSAIVSSVDKIPYGMRFIAKVLKDSLHEKFPDAGEDELLKIIGNLLYYRYMNPAIVAPDAFDIIDLSAGGQLTTDQRRNLGSIAKMLQHAASNKMFLGDNAHLSIINEYLSQSYQKFRRFFQTACDVPELQDKFNVDEYSDLVTLTKPVIYISIGEIINTHTLLLDHQDAIAPEHNDPIHELLDDLGEVPTIESLIGESSGNLNDPNKEALAKTEVSLTLTNKFDVPGDENAEMDARTILLNTKRLIVDVIRFQPGETLTEILETPATSEQEAEHQRAMQRRAIRDAKTPDKMKKSKSVKEDSNLTLQEKKEKIQTGLKKLTELGTVDPKNKYQELINDIARDIRNQRRYRQRRKAELVKLQQTYAALNSKATFYGEQVDYYKSYIKTCLDNLASKGKVSKKPREMKGKKSKKISLKYTAARLHEKGVLLEIEDLQVNQFKNVIFEISPTEEVGDFEVKAKFMGVQMETFMLHYQDLLQLQYEGVAVMKLFDRAKVNVNLLIFLLNKKFYGK",
# CAPZB_HUMAN
"MSDQQLDCALDLMRRLPPQQIEKNLSDLIDLVPSLCEDLLSSVDQPLKIARDKVVGKDYLLCDYNRDGDSYRSPWSNKYDPPLEDGAMPSARLRKLEVEANNAFDQYRDLYFEGGVSSVYLWDLDHGFAGVILIKKAGDGSKKIKGCWDSIHVVEVQEKSSGRTAHYKLTSTVMLWLQTNKSGSGTMNLGGSLTRQMEKDETVSDCSPHIANIGRLVEDMENKIRSTLNEIYFGKTKDIVNGLRSVQTFADKSKQEALKNDLVEALKRKQQC",
# IF1AX_HUMAN
"MPKNKGKGGKNRRRGKNENESEKRELVFKEDGQEYAQVIKMLGNGRLEAMCFDGVKRLCHIRGKLRKKVWINTSDIILVGLRDYQDNKADVILKYNADEARSLKAYGELPEHAKINETDTFGPGDDDEIQFDDIGDDDEDIDDI",
# GBRA2_HUMAN
"MKTKLNIYNMQFLLFVFLVWDPARLVLANIQEDEAKNNITIFTRILDRLLDGYDNRLRPGLGDSITEVFTNIYVTSFGPVSDTDMEYTIDVFFRQKWKDERLKFKGPMNILRLNNLMASKIWTPDTFFHNGKKSVAHNMTMPNKLLRIQDDGTLLYTMRLTVQAECPMHLEDFPMDAHSCPLKFGSYAYTTSEVTYIWTYNASDSVQVAPDGSRLNQYDLLGQSIGKETIKSSTGEYTVMTAHFHLKRKIGYFVIQTYLPCIMTVILSQVSFWLNRESVPARTVFGVTTVLTMTTLSISARNSLPKVAYATAMDWFIAVCYAFVFSALIEFATVNYFTKRGWAWDGKSVVNDKKKEKASVMIQNNAYAVAVANYAPNLSKDPVLSTISKSATTPEPNKKPENKPAEAKKTFNSVSKIDRMSRIVFPVLFGTFNLVYWATYLNREPVLGVSP",
# GBRB2_HUMAN
"MWRVRKRGYFGIWSFPLIIAAVCAQSVNDPSNMSLVKETVDRLLKGYDIRLRPDFGGPPVAVGMNIDIASIDMVSEVNMDYTLTMYFQQAWRDKRLSYNVIPLNLTLDNRVADQLWVPDTYFLNDKKSFVHGVTVKNRMIRLHPDGTVLYGLRITTTAACMMDLRRYPLDEQNCTLEIESYGYTTDDIEFYWRGDDNAVTGVTKIELPQFSIVDYKLITKKVVFSTGSYPRLSLSFKLKRNIGYFILQTYMPSILITILSWVSFWINYDASAARVALGITTVLTMTTINTHLRETLPKIPYVKAIDMYLMGCFVFVFMALLEYALVNYIFFGRGPQRQKKAAEKAASANNEKMRLDVNKIFYKDIKQNGTQYRSLWDPTGNLSPTRRTTNYDFSLYTMDPHENILLSTLEIKNEMATSEAVMGLGDPRSTMLAYDASSIQYRKAGLPRHSFGRNALERHVAQKKSRLRRRASQLKITIPDLTDVNAIDRWSRIFFPVVFSFFNIVYWLYYVN",
# 5HT5A_HUMAN
"MDLPVNLTSFSLSTPSPLETNHSLGKDDLRPSSPLLSVFGVLILTLLGFLVAATFAWNLLVLATILRVRTFHRVPHNLVASMAVSDVLVAALVMPLSLVHELSGRRWQLGRRLCQLWIACDVLCCTASIWNVTAIALDRYWSITRHMEYTLRTRKCVSNVMIALTWALSAVISLAPLLFGWGETYSEGSEECQVSREPSYAVFSTVGAFYLPLCVVLFVYWKIYKAAKFRVGSRKTNSVSPISEAVEVKDSAKQPQMVFTVRHATVTFQPEGDTWREQKEQRAALMVGILIGVFVLCWIPFFLTELISPLCSCDIPAIWKSIFLWLGYSNSFFNPLIYTAFNKNYNSAFKNFFSRQH",
# P2RY1_HUMAN
"MTEVLWPAVPNGTDAAFLAGPGSSWGNSTVASTAAVSSSFKCALTKTGFQFYYLPAVYILVFIIGFLGNSVAIWMFVFHMKPWSGISVYMFNLALADFLYVLTLPALIFYYFNKTDWIFGDAMCKLQRFIFHVNLYGSILFLTCISAHRYSGVVYPLKSLGRLKKKNAICISVLVWLIVVVAISPILFYSGTGVRKNKTITCYDTTSDEYLRSYFIYSMCTTVAMFCVPLVLILGCYGLIVRALIYKDLDNSPLRRKSIYLVIIVLTVFAVSYIPFHVMKTMNLRARLDFQTPAMCAFNDRVYATYQVTRGLASLNSCVDPILYFLAGDTFRRRLSRATRKASRRSEANLQSKSEDMTLNILPEFKQNGDTSL",
# RL29_HUMAN
"MAKSKNHTTHNQSRKWHRNGIKKPRSQRYESLKGVDPKFLRNMRFAKKHNKKGLKKMQANNAKAMSARAEAIKALVKPKEVKPKIPKGVSRKLDRLAYIAHPKLGKRARARIAKGLRLCRPKAKAKAKAKDQTKAQAAAPASVPAQAPKRTQAPTKASE",
# NPTX2_HUMAN
"MLALLAASVALAVAAGAQDSPAPGSRFVCTALPPEAVHAGCPLPAMPMQGGAQSPEEELRAAVLQLRETVVQQKETLGAQREAIRELTGKLARCEGLAGGKARGAGATGKDTMGDLPRDPGHVVEQLSRSLQTLKDRLESLEHQLRANVSNAGLPGDFREVLQQRLGELERQLLRKVAELEDEKSLLHNETSAHRQKTESTLNALLQRVTELERGNSAFKSPDAFKVSLPLRTNYLYGKIKKTLPELYAFTICLWLRSSASPGIGTPFSYAVPGQANEIVLIEWGNNPIELLINDKVAQLPLFVSDGKWHHICVTWTTRDGMWEAFQDGEKLGTGENLAPWHPIKPGGVLILGQEQDTVGGRFDATQAFVGELSQFNIWDRVLRAQEIVNIANCSTNMPGNIIPWVDNNVDVFGGASKWPVETCEERLLDL",
# SC6A8_HUMAN
"MAKKSAENGIYSVSGDEKKGPLIAPGPDGAPAKGDGPVGLGTPGGRLAVPPRETWTRQMDFIMSCVGFAVGLGNVWRFPYLCYKNGGGVFLIPYVLIALVGGIPIFFLEISLGQFMKAGSINVWNICPLFKGLGYASMVIVFYCNTYYIMVLAWGFYYLVKSFTTTLPWATCGHTWNTPDCVEIFRHEDCANASLANLTCDQLADRRSPVIEFWENKVLRLSGGLEVPGALNWEVTLCLLACWVLVYFCVWKGVKSTGKIVYFTATFPYVVLVVLLVRGVLLPGALDGIIYYLKPDWSKLGSPQVWIDAGTQIFFSYAIGLGALTALGSYNRFNNNCYKDAIILALINSGTSFFAGFVVFSILGFMAAEQGVHISKVAESGPGLAFIAYPRAVTLMPVAPLWAALFFFMLLLLGLDSQFVGVEGFITGLLDLLPASYYFRFQREISVALCCALCFVIDLSMVTDGGMYVFQLFDYYSASGTTLLWQAFWECVVVAWVYGADRFMDDIACMIGYRPCPWMKWCWSFFTPLVCMGIFIFNVVYYEPLVYNNTYVYPWWGEAMGWAFALSSMLCVPLHLLGCLLRAKGTMAERWQHLTQPIWGLHHLEYRAQDADVRGLTTLTPVSESSKVVVVESVM",
# KCNJ4_HUMAN
"MHGHSRNGQAHVPRRKRRNRFVKKNGQCNVYFANLSNKSQRYMADIFTTCVDTRWRYMLMIFSAAFLVSWLFFGLLFWCIAFFHGDLEASPGVPAAGGPAAGGGGAAPVAPKPCIMHVNGFLGAFLFSVETQTTIGYGFRCVTEECPLAVIAVVVQSIVGCVIDSFMIGTIMAKMARPKKRAQTLLFSHHAVISVRDGKLCLMWRVGNLRKSHIVEAHVRAQLIKPYMTQEGEYLPLDQRDLNVGYDIGLDRIFLVSPIIIVHEIDEDSPLYGMGKEELESEDFEIVVILEGMVEATAMTTQARSSYLASEILWGHRFEPVVFEEKSHYKVDYSRFHKTYEVAGTPCCSARELQESKITVLPAPPPPPSAFCYENELALMSQEEEEMEEEAAAAAAVAAGLGLEAGSKEEAGIIRMLEFGSHLDLERMQASLPLDNISYRRESAI",
# KCNJ6_HUMAN
"MAKLTESMTNVLEGDSMDQDVESPVAIHQPKLPKQARDDLPRHISRDRTKRKIQRYVRKDGKCNVHHGNVRETYRYLTDIFTTLVDLKWRFNLLIFVMVYTVTWLFFGMIWWLIAYIRGDMDHIEDPSWTPCVTNLNGFVSAFLFSIETETTIGYGYRVITDKCPEGIILLLIQSVLGSIVNAFMVGCMFVKISQPKKRAETLVFSTHAVISMRDGKLCLMFRVGDLRNSHIVEASIRAKLIKSKQTSEGEFIPLNQTDINVGYYTGDDRLFLVSPLIISHEINQQSPFWEISKAQLPKEELEIVVILEGMVEATGMTCQARSSYITSEILWGYRFTPVLTLEDGFYEVDYNSFHETYETSTPSLSAKELAELASRAELPLSWSVSSKLNQHAELETEEEEKNLEEQTERNGDVANLENESKV",
# GRIA4_HUMAN
"MRIISRQIVLLFSGFWGLAMGAFPSSVQIGGLFIRNTDQEYTAFRLAIFLHNTSPNASEAPFNLVPHVDNIETANSFAVTNAFCSQYSRGVFAIFGLYDKRSVHTLTSFCSALHISLITPSFPTEGESQFVLQLRPSLRGALLSLLDHYEWNCFVFLYDTDRGYSILQAIMEKAGQNGWHVSAICVENFNDVSYRQLLEELDRRQEKKFVIDCEIERLQNILEQIVSVGKHVKGYHYIIANLGFKDISLERFIHGGANVTGFQLVDFNTPMVIKLMDRWKKLDQREYPGSETPPKYTSALTYDGVLVMAETFRSLRRQKIDISRRGNAGDCLANPAAPWGQGIDMERTLKQVRIQGLTGNVQFDHYGRRVNYTMDVFELKSTGPRKVGYWNDMDKLVLIQDVPTLGNDTAAIENRTVVVTTIMESPYVMYKKNHEMFEGNDKYEGYCVDLASEIAKHIGIKYKIAIVPDGKYGARDADTKIWNGMVGELVYGKAEIAIAPLTITLVREEVIDFSKPFMSLGISIMIKKPQKSKPGVFSFLDPLAYEIWMCIVFAYIGVSVVLFLVSRFSPYEWHTEEPEDGKEGPSDQPPNEFGIFNSLWFSLGAFMQQGCDISPRSLSGRIVGGVWWFFTLIIISSYTANLAAFLTVERMVSPIESAEDLAKQTEIAYGTLDSGSTKEFFRRSKIAVYEKMWTYMRSAEPSVFTRTTAEGVARVRKSKGKFAFLLESTMNEYIEQRKPCDTMKVGGNLDSKGYGVATPKGSSLRTPVNLAVLKLSEAGVLDKLKNKWWYDKGECGPKDSGSKDKTSALSLSNVAGVFYILVGGLGLAMLVALIEFCYKSRAEAKRMKLTFSEAIRNKARLSITGSVGENGRVLTPDCPKAVHTGTAIRQSSGLAVIASDLP",
# S6A11_HUMAN
"MTAEKALPLGNGKAAEEARESEAPGGGCSSGGAAPARHPRVKRDKAVHERGHWNNKVEFVLSVAGEIIGLGNVWRFPYLCYKNGGGAFLIPYVVFFICCGIPVFFLETALGQFTSEGGITCWRKVCPLFEGIGYATQVIEAHLNVYYIIILAWAIFYLSNCFTTELPWATCGHEWNTENCVEFQKLNVSNYSHVSLQNATSPVMEFWEHRVLAISDGIEHIGNLRWELALCLLAAWTICYFCIWKGTKSTGKVVYVTATFPYIMLLILLIRGVTLPGASEGIKFYLYPDLSRLSDPQVWVDAGTQIFFSYAICLGCLTALGSYNNYNNNCYRDCIMLCCLNSGTSFVAGFAIFSVLGFMAYEQGVPIAEVAESGPGLAFIAYPKAVTMMPLSPLWATLFFMMLIFLGLDSQFVCVESLVTAVVDMYPKVFRRGYRRELLILALSVISYFLGLVMLTEGGMYIFQLFDSYAASGMCLLFVAIFECICIGWVYGSNRFYDNIEDMIGYRPPSLIKWCWMIMTPGICAGIFIFFLIKYKPLKYNNIYTYPAWGYGIGWLMALSSMLCIPLWICITVWKTEGTLPEKLQKLTTPSTDLKMRGKLGVSPRMVTVNDCDAKLKSDGTIAAITEKETHF",
# SC6A9_HUMAN
"MSGGDTRAAIARPRMAAAHGPVAPSSPEQVTLLPVQRSFFLPPFSGATPSTSLAESVLKVWHGAYNSGLLPQLMAQHSLAMAQNGAVPSEATKRDQNLKRGNWGNQIEFVLTSVGYAVGLGNVWRFPYLCYRNGGGAFMFPYFIMLIFCGIPLFFMELSFGQFASQGCLGVWRISPMFKGVGYGMMVVSTYIGIYYNVVICIAFYYFFSSMTHVLPWAYCNNPWNTHDCAGVLDASNLTNGSRPAALPSNLSHLLNHSLQRTSPSEEYWRLYVLKLSDDIGNFGEVRLPLLGCLGVSWLVVFLCLIRGVKSSGKVVYFTATFPYVVLTILFVRGVTLEGAFDGIMYYLTPQWDKILEAKVWGDAASQIFYSLGCAWGGLITMASYNKFHNNCYRDSVIISITNCATSVYAGFVIFSILGFMANHLGVDVSRVADHGPGLAFVAYPEALTLLPISPLWSLLFFFMLILLGLGTQFCLLETLVTAIVDEVGNEWILQKKTYVTLGVAVAGFLLGIPLTSQAGIYWLLLMDNYAASFSLVVISCIMCVAIMYIYGHRNYFQDIQMMLGFPPPLFFQICWRFVSPAIIFFILVFTVIQYQPITYNHYQYPGWAVAIGFLMALSSVLCIPLYAMFRLCRTDGDTLLQRLKNATKPSRDWGPALLEHRTGRYAPTIAPSPEDGFEVQPLHPDKAQIPIVGSNGSSRLQDSRI",
# GLRB_HUMAN
"MKFLLTTAFLILISLWVEEAYSKEKSSKKGKGKKKQYLCPSQQSAEDLARVPANSTSNILNRLLVSYDPRIRPNFKGIPVDVVVNIFINSFGSIQETTMDYRVNIFLRQKWNDPRLKLPSDFRGSDALTVDPTMYKCLWKPDLFFANEKSANFHDVTQENILLFIFRDGDVLVSMRLSITLSCPLDLTLFPMDTQRCKMQLESFGYTTDDLRFIWQSGDPVQLEKIALPQFDIKKEDIEYGNCTKYYKGTGYYTCVEVIFTLRRQVGFYMMGVYAPTLLIVVLSWLSFWINPDASAARVPLGIFSVLSLASECTTLAAELPKVSYVKALDVWLIACLLFGFASLVEYAVVQVMLNNPKRVEAEKARIAKAEQADGKGGNVAKKNTVNGTGTPVHISTLQVGETRCKKVCTSKSDLRSNDFSIVGSLPRDFELSNYDCYGKPIEVNNGLGKSQAKNNKKPPPAKPVIPTAAKRIDLYARALFPFCFLFFNVIYWSIYL",
# GBRA4_HUMAN
"MVSAKKVPAIALSAGVSFALLRFLCLAVCLNESPGQNQKEEKLCTENFTRILDSLLDGYDNRLRPGFGGPVTEVKTDIYVTSFGPVSDVEMEYTMDVFFRQTWIDKRLKYDGPIEILRLNNMMVTKVWTPDTFFRNGKKSVSHNMTAPNKLFRIMRNGTILYTMRLTISAECPMRLVDFPMDGHACPLKFGSYAYPKSEMIYTWTKGPEKSVEVPKESSSLVQYDLIGQTVSSETIKSITGEYIVMTVYFHLRRKMGYFMIQTYIPCIMTVILSQVSFWINKESVPARTVFGITTVLTMTTLSISARHSLPKVSYATAMDWFIAVCFAFVFSALIEFAAVNYFTNIQMEKAKRKTSKPPQEVPAAPVQREKHPEAPLQNTNANLNMRKRTNALVHSESDVGNRTEVGNHSSKSSTVVQESSKGTPRSYLASSPNPFSRANAAETISAARALPSASPTSIRTGYMPRKASVGSASTRHVFGSRLQRIKTTVNTIGATGKLSATPPPSAPPPSGSGTSKIDKYARILFPVTFGAFNMVYWVVYLSKDTMEKSESLM",
# PP2BC_HUMAN
"MSGRRFHLSTTDRVIKAVPFPPTQRLTFKEVFENGKPKVDVLKNHLVKEGRLEEEVALKIINDGAAILRQEKTMIEVDAPITVCGDIHGQFFDLMKLFEVGGSPSNTRYLFLGDYVDRGYFSIECVLYLWSLKINHPKTLFLLRGNHECRHLTDYFTFKQECRIKYSEQVYDACMETFDCLPLAALLNQQFLCVHGGMSPEITSLDDIRKLDRFTEPPAFGPVCDLLWSDPSEDYGNEKTLEHYTHNTVRGCSYFYSYPAVCEFLQNNNLLSIIRAHEAQDAGYRMYRKSQATGFPSLITIFSAPNYLDVYNNKAAVLKYENNVMNIRQFNCSPHPYWLPNFMDVFTWSLPFVGEKVTEMLVNVLNICSDDELISDDEAEGSTTVRKEIIRNKIRAIGKMARVFSILRQESESVLTLKGLTPTGTLPLGVLSGGKQTIETATVEAVEAREAIRGFSLQHKIRSFEEARGLDRINERMPPRKDSIHAGGPMKSVTSAHSHAAHRSDQGKKAHS",
# KCNC1_HUMAN
"MGQGDESERIVINVGGTRHQTYRSTLRTLPGTRLAWLAEPDAHSHFDYDPRADEFFFDRHPGVFAHILNYYRTGKLHCPADVCGPLYEEELAFWGIDETDVEPCCWMTYRQHRDAEEALDSFGGAPLDNSADDADADGPGDSGDGEDELEMTKRLALSDSPDGRPGGFWRRWQPRIWALFEDPYSSRYARYVAFASLFFILVSITTFCLETHERFNPIVNKTEIENVRNGTQVRYYREAETEAFLTYIEGVCVVWFTFEFLMRVIFCPNKVEFIKNSLNIIDFVAILPFYLEVGLSGLSSKAAKDVLGFLRVVRFVRILRIFKLTRHFVGLRVLGHTLRASTNEFLLLIIFLALGVLIFATMIYYAERIGAQPNDPSASEHTHFKNIPIGFWWAVVTMTTLGYGDMYPQTWSGMLVGALCALAGVLTIAMPVPVIVNNFGMYYSLAMAKQKLPKKKKKHIPRPPQLGSPNYCKSVVNSPHHSTQSDTCPLAQEEILEINRAGRKPLRGMSI",
# KCNJ3_HUMAN
"MSALRRKFGDDYQVVTTSSSGSGLQPQGPGQDPQQQLVPKKKRQRFVDKNGRCNVQHGNLGSETSRYLSDLFTTLVDLKWRWNLFIFILTYTVAWLFMASMWWVIAYTRGDLNKAHVGNYTPCVANVYNFPSAFLFFIETEATIGYGYRYITDKCPEGIILFLFQSILGSIVDAFLIGCMFIKMSQPKKRAETLMFSEHAVISMRDGKLTLMFRVGNLRNSHMVSAQIRCKLLKSRQTPEGEFLPLDQLELDVGFSTGADQLFLVSPLTICHVIDAKSPFYDLSQRSMQTEQFEIVVILEGIVETTGMTCQARTSYTEDEVLWGHRFFPVISLEEGFFKVDYSQFHATFEVPTPPYSVKEQEEMLLMSSPLIAPAITNSKERHNSVECLDGLDDITTKLPSKLQKITGREDFPKKLLRMSSTTSEKAYSLGDLPMKLQRISSVPGNSEEKLVSKTTKMLSDPMSQSVADLPPKLQKMAGGAARMEGNLPAKLRKMNSDRFT",
# EAA4_HUMAN
"MSSHGNSLFLRESGQRLGRVGWLQRLQESLQQRALRTRLRLQTMTLEHVLRFLRRNAFILLTVSAVVIGVSLAFALRPYQLTYRQIKYFSFPGELLMRMLQMLVLPLIVSSLVTGMASLDNKATGRMGMRAAVYYMVTTIIAVFIGILMVTIIHPGKGSKEGLHREGRIETIPTADAFMDLIRNMFPPNLVEACFKQFKTQYSTRVVTRTMVRTENGSEPGASMPPPFSVENGTSFLENVTRALGTLQEMLSFEETVPVPGSANGINALGLVVFSVAFGLVIGGMKHKGRVLRDFFDSLNEAIMRLVGIIIWYAPVGILFLIAGKILEMEDMAVLGGQLGMYTLTVIVGLFLHAGIVLPLIYFLVTHRNPFPFIGGMLQALITAMGTSSSSATLPITFRCLEEGLGVDRRITRFVLPVGATVNMDGTALYEALAAIFIAQVNNYELNLGQITTISITATAASVGAAGIPQAGLVTMVIVLTSVGLPTEDITLIIAVDWFLDRLRTMTNVLGDSIGAAVIEHLSQRELELQEAELTLPSLGKPYKSLMAQEKGASRGRGGNESAM",
# TRPC1_HUMAN
"MMAALYPSTDLSGASSSSLPSSPSSSSPNEVMALKDVREVKEENTLNEKLFLLACDKGDYYMVKKILEENSSGDLNINCVDVLGRNAVTITIENENLDILQLLLDYGCQSADALLVAIDSEVVGAVDILLNHRPKRSSRPTIVKLMERIQNPEYSTTMDVAPVILAAHRNNYEILTMLLKQDVSLPKPHAVGCECTLCSAKNKKDSLRHSRFRLDIYRCLASPALIMLTEEDPILRAFELSADLKELSLVEVEFRNDYEELARQCKMFAKDLLAQARNSRELEVILNHTSSDEPLDKRGLLEERMNLSRLKLAIKYNQKEFVSQSNCQQFLNTVWFGQMSGYRRKPTCKKIMTVLTVGIFWPVLSLCYLIAPKSQFGRIIHTPFMKFIIHGASYFTFLLLLNLYSLVYNEDKKNTMGPALERIDYLLILWIIGMIWSDIKRLWYEGLEDFLEESRNQLSFVMNSLYLATFALKVVAHNKFHDFADRKDWDAFHPTLVAEGLFAFANVLSYLRLFFMYTTSSILGPLQISMGQMLQDFGKFLGMFLLVLFSFTIGLTQLYDKGYTSKEQKDCVGIFCEQQSNDTFHSFIGTCFALFWYIFSLAHVAIFVTRFSYGEELQSFVGAVIVGTYNVVVVIVLTKLLVAMLHKSFQLIANHEDKEWKFARAKLWLSYFDDKCTLPPPFNIIPSPKTICYMISSLSKWICSHTSKGKVKRQNSLKEWRNLKQKRDENYQKVMCCLVHRYLTSMRQKMQSTDQATVENLNELRQDLSKFRNEIRDLLGFRTSKYAMFYPRN",
# MRP_HUMAN
"MGSQSSKAPRGDVTAEEAAGASPAKANGQENGHVKSNGDLSPKGEGESPPVNGTDEAAGATGDAIEPAPPSQGAEAKGEVPPKETPKKKKKFSFKKPFKLSGLSFKRNRKEGGGDSSASSPTEEEQEQGEIGACSDEGTAQEGKAAATPESQEPQAKGAEASAASEEEAGPQATEPSTPSGPESGPTPASAEQNE",
# NPY2R_HUMAN
"MGPIGAEADENQTVEEMKVEQYGPQTTPRGELVPDPEPELIDSTKLIEVQVVLILAYCSIILLGVIGNSLVIHVVIKFKSMRTVTNFFIANLAVADLLVNTLCLPFTLTYTLMGEWKMGPVLCHLVPYAQGLAVQVSTITLTVIALDRHRCIVYHLESKISKRISFLIIGLAWGISALLASPLAIFREYSLIEIIPDFEIVACTEKWPGEEKSIYGTVYSLSSLLILYVLPLGIISFSYTRIWSKLKNHVSPGAANDHYHQRRQKTTKMLVCVVVVFAVSWLPLHAFQLAVDIDSQVLDLKEYKLIFTVFHIIAMCSTFANPLLYGWMNSNYRKAFLSAFRCEQRLDAIHSEVSVTFKAKKNLEVRKNSGPNDSFTEATNV",
# RL34_HUMAN
"MVQRLTYRRRLSYNTASNKTRLSRTPGNRIVYLYTKKVGKAPKSACGVCPGRLRGVRAVRPKVLMRLSKTKKHVSRAYGGSMCAKCVRDRIKRAFLIEEQKIVVKVLKAQAQSQKAK",
# LMAN1_HUMAN
"MAGSRQRGLRARVRPLFCALLLSLGRFVRGDGVGGDPAVALPHRRFEYKYSFKGPHLVQSDGTVPFWAHAGNAIPSSDQIRVAPSLKSQRGSVWTKTKAAFENWEVEVTFRVTGRGRIGADGLAIWYAENQGLEGPVFGSADLWNGVGIFFDSFDNDGKKNNPAIVIIGNNGQIHYDHQNDGASQALASCQRDFRNKPYPVRAKITYYQNTLTVMINNGFTPDKNDYEFCAKVENMIIPAQGHFGISAATGGLADDHDVLSFLTFQLTEPGKEPPTPDKEISEKEKEKYQEEFEHFQQELDKKKEEFQKGHPDLQGQPAEEIFESVGDRELRQVFEGQNRIHLEIKQLNRQLDMILDEQRRYVSSLTEEISKRGAGMPGQHGQITQQELDTVVKTQHEILRQVNEMKNSMSETVRLVSGMQHPGSAGGVYETTQHFIDIKEHLHIVKRDIDNLVQRNMPSNEKPKCPELPPFPSCLSTVHFIIFVVVQTVLFIGYIMYRSQQEAAAKKFF",
# ARRB1_HUMAN
"MGDKGTRVFKKASPNGKLTVYLGKRDFVDHIDLVDPVDGVVLVDPEYLKERRVYVTLTCAFRYGREDLDVLGLTFRKDLFVANVQSFPPAPEDKKPLTRLQERLIKKLGEHAYPFTFEIPPNLPCSVTLQPGPEDTGKACGVDYEVKAFCAENLEEKIHKRNSVRLVIRKVQYAPERPGPQPTAETTRQFLMSDKPLHLEASLDKEIYYHGEPISVNVHVTNNTNKTVKKIKISVRQYADICLFNTAQYKCPVAMEEADDTVAPSSTFCKVYTLTPFLANNREKRGLALDGKLKHEDTNLASSTLLREGANREILGIIVSYKVKVKLVVSRGGLLGDLASSDVAVELPFTLMHPKPKEEPPHREVPENETPVDTNLIELDTNDDDIVFEDFARQRLKGMKDDKEEEEDGTGSPQLNNR",
# EFTU_HUMAN
"MTTMAAATLLRATPHFSGLAAGRTFLLQGLLRLLKAPALPLLCRGLAVEAKKTYVRDKPHVNVGTIGHVDHGKTTLTAAITKILAEGGGAKFKKYEEIDNAPEERARGITINAAHVEYSTAARHYAHTDCPGHADYVKNMITGTAPLDGCILVVAANDGPMPQTREHLLLARQIGVEHVVVYVNKADAVQDSEMVELVELEIRELLTEFGYKGEETPVIVGSALCALEGRDPELGLKSVQKLLDAVDTYIPVPARDLEKPFLLPVEAVYSVPGRGTVVTGTLERGILKKGDECELLGHSKNIRTVVTGIEMFHKSLERAEAGDNLGALVRGLKREDLRRGLVMVKPGSIKPHQKVEAQVYILSKEEGGRHKPFVSHFMPVMFSLTWDMACRIILPPEKELAMPGEDLKFNLILRQPMILEKGQRFTLRDGNRTIGTGLVTNTLAMTEEEKNIKWG",
# AMPH_HUMAN
"MADIKTGIFAKNVQKRLNRAQEKVLQKLGKADETKDEQFEEYVQNFKRQEAEGTRLQRELRGYLAAIKGMQEASMKLTESLHEVYEPDWYGREDVKMVGEKCDVLWEDFHQKLVDGSLLTLDTYLGQFPDIKNRIAKRSRKLVDYDSARHHLEALQSSKRKDESRISKAEEEFQKAQKVFEEFNVDLQEELPSLWSRRVGFYVNTFKNVSSLEAKFHKEIAVLCHKLYEVMTKLGDQHADKAFTIQGAPSDSGPLRIAKTPSPPEEPSPLPSPTASPNHTLAPASPAPARPRSPSQTRKGPPVPPLPKVTPTKELQQENIISFFEDNFVPEISVTTPSQNEVPEVKKEETLLDLDFDPFKPEVTPAGSAGVTHSPMSQTLPWDLWTTSTDLVQPASGGSFNGFTQPQDTSLFTMQTDQSMICNLAESEQAPPTEPKAEEPLAAVTPAVGLDLGMDTRAEEPVEEAVIIPGADADAAVGTLVSAAEGAPGEEAEAEKATVPAGEGVSLEEAKIGTETTEGAESAQPEAEELEATVPQEKVIPSVVIEPASNHEEEGENEITIGAEPKETTEDAAPPGPTSETPELATEQKPIQDPQPTPSAPAMGAADQLASAREASQELPPGFLYKVETLHDFEAANSDELTLQRGDVVLVVPSDSEADQDAGWLVGVKESDWLQYRDLATYKGLFPENFTRRLD",
# NUMB_HUMAN
"MNKLRQSFRRKKDVYVPEASRPHQWQTDEEGVRTGKCSFPVKYLGHVEVDESRGMHICEDAVKRLKAERKFFKGFFGKTGKKAVKAVLWVSADGLRVVDEKTKDLIVDQTIEKVSFCAPDRNFDRAFSYICRDGTTRRWICHCFMAVKDTGERLSHAVGCAFAACLERKQKREKECGVTATFDASRTTFTREGSFRVTTATEQAEREEIMKQMQDAKKAETDKIVVGSSVAPGNTAPSPSSPTSPTSDATTSLEMNNPHAIPRRHAPIEQLARQGSFRGFPALSQKMSPFKRQLSLRINELPSTMQRKTDFPIKNAVPEVEGEAESISSLCSQITNAFSTPEDPFSSAPMTKPVTVVAPQSPTFQANGTDSAFHVLAKPAHTALAPVAMPVRETNPWAHAPDAANKEIAATCSGTEWGQSSGAASPGLFQAGHRRTPSEADRWLEEVSKSVRAQQPQASAAPLQPVLQPPPPTAISQPASPFQGNAFLTSQPVPVGVVPALQPAFVPAQSYPVANGMPYPAPNVPVVGITPSQMVANVFGTAGHPQAAHPHQSPSLVRQQTFPHYEASSATTSPFFKPPAQHLNGSAAFNGVDDGRLASADRHTEVPTGTCPVDPFEAQWAALENKSKQRTNPSPTNPFSSDLQKTFEIEL",
# PSN1_HUMAN
"MTELPAPLSYFQNAQMSEDNHLSNTVRSQNDNRERQEHNDRRSLGHPEPLSNGRPQGNSRQVVEQDEEEDEELTLKYGAKHVIMLFVPVTLCMVVVVATIKSVSFYTRKDGQLIYTPFTEDTETVGQRALHSILNAAIMISVIVVMTILLVVLYKYRCYKVIHAWLIISSLLLLFFFSFIYLGEVFKTYNVAVDYITVALLIWNFGVVGMISIHWKGPLRLQQAYLIMISALMALVFIKYLPEWTAWLILAVISVYDLVAVLCPKGPLRMLVETAQERNETLFPALIYSSTMVWLVNMAEGDPEAQRRVSKNSKYNAESTERESQDTVAENDDGGFSEEWEAQRDSHLGPHRSTPESRAAVQELSSSILAGEDPEERGVKLGLGDFIFYSVLVGKASATASGDWNTTIACFVAILIGLCLTLLLLAIFKKALPALPISITFGLVFYFATDYLVQPFMDQLAFHQFYI",
# EI2BB_HUMAN
"MPGSAAKGSELSERIESFVETLKRGGGPRSSEEMARETLGLLRQIITDHRWSNAGELMELIRREGRRMTAAQPSETTVGNMVRRVLKIIREEYGRLHGRSDESDQQESLHKLLTSGGLNEDFSFHYAQLQSNIIEAINELLVELEGTMENIAAQALEHIHSNEVIMTIGFSRTVEAFLKEAARKRKFHVIVAECAPFCQGHEMAVNLSKAGIETTVMTDAAIFAVMSRVNKVIIGTKTILANGALRAVTGTHTLALAAKHHSTPLIVCAPMFKLSPQFPNEEDSFHKFVAPEEVLPFTEGDILEKVSVHCPVFDYVPPELITLFISNIGGNAPSYIYRLMSELYHPDDHVL",
# RGS4_HUMAN
"MCKGLAGLPASCLRSAKDMKHRLGFLLQKSDSCEHNSSHNKKDKVVICQRVSQEEVKKWAESLENLISHECGLAAFKAFLKSEYSEENIDFWISCEEYKKIKSPSKLSPKAKKIYNEFISVQATKEVNLDSCTREETSRNMLEPTITCFDEAQKKIFNLMEKDSYRRFLKSRFYLDLVNPSSCGAEKQKGAKSSADCASLVPQCA",
# RGS7_HUMAN
"MAQGNNYGQTSNGVADESPNMLVYRKMEDVIARMQDEKNGIPIRTVKSFLSKIPSVFSGSDIVQWLIKNLTIEDPVEALHLGTLMAAHGYFFPISDHVLTLKDDGTFYRFQTPYFWPSNCWEPENTDYAVYLCKRTMQNKARLELADYEAESLARLQRAFARKWEFIFMQAEAQAKVDKKRDKIERKILDSQERAFWDVHRPVPGCVNTTEVDIKKSSRMRNPHKTRKSVYGLQNDIRSHSPTHTPTPETKPPTEDELQQQIKYWQIQLDRHRLKMSKVADSLLSYTEQYLEYDPFLLPPDPSNPWLSDDTTFWELEASKEPSQQRVKRWGFGMDEALKDPVGREQFLKFLESEFSSENLRFWLAVEDLKKRPIKEVPSRVQEIWQEFLAPGAPSAINLDSKSYDKTTQNVKEPGRYTFEDAQEHIYKLMKSDSYPRFIRSSAYQELLQAKKKSGNSMDRRTSFEKFAQNVGRNIPIFPCHKNCTPTLRASTNLL",
# PSN2_HUMAN
"MLTFMASDSEEEVCDERTSLMSAESPTPRSCQEGRQGPEDGENTAQWRSQENEEDGEEDPDRYVCSGVPGRPPGLEEELTLKYGAKHVIMLFVPVTLCMIVVVATIKSVRFYTEKNGQLIYTPFTEDTPSVGQRLLNSVLNTLIMISVIVVMTIFLVVLYKYRCYKFIHGWLIMSSLMLLFLFTYIYLGEVLKTYNVAMDYPTLLLTVWNFGAVGMVCIHWKGPLVLQQAYLIMISALMALVFIKYLPEWSAWVILGAISVYDLVAVLCPKGPLRMLVETAQERNEPIFPALIYSSAMVWTVGMAKLDPSSQGALQLPYDPEMEEDSYDSFGEPSYPEVFEPPLTGYPGEELEEEEERGVKLGLGDFIFYSVLVGKAAATGSGDWNTTLACFVAILIGLCLTLLLLAVFKKALPALPISITFGLIFYFSTDNLVRPFMDTLASHQLYI",
# TSC2_HUMAN
"MAKPTSKDSGLKEKFKILLGLGTPRPNPRSAEGKQTEFIITAEILRELSMECGLNNRIRMIGQICEVAKTKKFEEHAVEALWKAVADLLQPERPLEARHAVLALLKAIVQGQGERLGVLRALFFKVIKDYPSNEDLHERLEVFKALTDNGRHITYLEEELADFVLQWMDVGLSSEFLLVLVNLVKFNSCYLDEYIARMVQMICLLCVRTASSVDIEVSLQVLDAVVCYNCLPAESLPLFIVTLCRTINVKELCEPCWKLMRNLLGTHLGHSAIYNMCHLMEDRAYMEDAPLLRGAVFFVGMALWGAHRLYSLRNSPTSVLPSFYQAMACPNEVVSYEIVLSITRLIKKYRKELQVVAWDILLNIIERLLQQLQTLDSPELRTIVHDLLTTVEELCDQNEFHGSQERYFELVERCADQRPESSLLNLISYRAQSIHPAKDGWIQNLQALMERFFRSESRGAVRIKVLDVLSFVLLINRQFYEEELINSVVISQLSHIPEDKDHQVRKLATQLLVDLAEGCHTHHFNSLLDIIEKVMARSLSPPPELEERDVAAYSASLEDVKTAVLGLLVILQTKLYTLPASHATRVYEMLVSHIQLHYKHSYTLPIASSIRLQAFDFLLLLRADSLHRLGLPNKDGVVRFSPYCVCDYMEPERGSEKKTSGPLSPPTGPPGPAPAGPAVRLGSVPYSLLFRVLLQCLKQESDWKVLKLVLGRLPESLRYKVLIFTSPCSVDQLCSALCSMLSGPKTLERLRGAPEGFSRTDLHLAVVPVLTALISYHNYLDKTKQREMVYCLEQGLIHRCASQCVVALSICSVEMPDIIIKALPVLVVKLTHISATASMAVPLLEFLSTLARLPHLYRNFAAEQYASVFAISLPYTNPSKFNQYIVCLAHHVIAMWFIRCRLPFRKDFVPFITKGLRSNVLLSFDDTPEKDSFRARSTSLNERPKSLRIARPPKQGLNNSPPVKEFKESSAAEAFRCRSISVSEHVVRSRIQTSLTSASLGSADENSVAQADDSLKNLHLELTETCLDMMARYVFSNFTAVPKRSPVGEFLLAGGRTKTWLVGNKLVTVTTSVGTGTRSLLGLDSGELQSGPESSSSPGVHVRQTKEAPAKLESQAGQQVSRGARDRVRSMSGGHGLRVGALDVPASQFLGSATSPGPRTAPAAKPEKASAGTRVPVQEKTNLAAYVPLLTQGWAEILVRRPTGNTSWLMSLENPLSPFSSDINNMPLQELSNALMAAERFKEHRDTALYKSLSVPAASTAKPPPLPRSNTVASFSSLYQSSCQGQLHRSVSWADSAVVMEEGSPGEVPVLVEPPGLEDVEAALGMDRRTDAYSRSSSVSSQEEKSLHAEELVGRGIPIERVVSSEGGRPSVDLSFQPSQPLSKSSSSPELQTLQDILGDPGDKADVGRLSPEVKARSQSGTLDGESAAWSASGEDSRGQPEGPLPSSSPRSPSGLRPRGYTISDSAPSRRGKRVERDALKSRATASNAEKVPGINPSFVFLQLYHSPFFGDESNKPILLPNESQSFERSVQLLDQIPSYDTHKIAVLYVGEGQSNSELAILSNEHGSYRYTEFLTGLGRLIELKDCQPDKVYLGGLDVCGEDGQFTYCWHDDIMQAVFHIATLMPTKDVDKHRCDKKRHLGNDFVSIVYNDSGEDFKLGTIKGQFNFVHVIVTPLDYECNLVSLQCRKDMEGLVDTSVAKIVSDRNLPFVARQMALHANMASQVHHSRSNPTDIYPSKWIARLRHIKRLRQRICEEAAYSNPSLPLVHPPSHSKAPAQTPAEPTPGYEVGQRKRLISSVEDFTEFV",
# GSK3B_HUMAN
"MSGRPRTTSFAESCKPVQQPSAFGSMKVSRDKDGSKVTTVVATPGQGPDRPQEVSYTDTKVIGNGSFGVVYQAKLCDSGELVAIKKVLQDKRFKNRELQIMRKLDHCNIVRLRYFFYSSGEKKDEVYLNLVLDYVPETVYRVARHYSRAKQTLPVIYVKLYMYQLFRSLAYIHSFGICHRDIKPQNLLLDPDTAVLKLCDFGSAKQLVRGEPNVSYICSRYYRAPELIFGATDYTSSIDVWSAGCVLAELLLGQPIFPGDSGVDQLVEIIKVLGTPTREQIREMNPNYTEFKFPQIKAHPWTKVFRPRTPPEAIALCSRLLEYTPTARLTPLEACAHSFFDELRDPNVKLPNGRDTPALFNFTTQELSSNPPLATILIPPHARIQAAASTPTNATAASDANTGDRGQTNNAASASASNST",
# GNAQ_HUMAN
"MTLESIMACCLSEEAKEARRINDEIERQLRRDKRDARRELKLLLLGTGESGKSTFIKQMRIIHGSGYSDEDKRGFTKLVYQNIFTAMQAMIRAMDTLKIPYKYEHNKAHAQLVREVDVEKVSAFENPYVDAIKSLWNDPGIQECYDRRREYQLSDSTKYYLNDLDRVADPAYLPTQQDVLRVRVPTTGIIEYPFDLQSVIFRMVDVGGQRSERRKWIHCFENVTSIMFLVALSEYDQVLVESDNENRMEESKALFRTIITYPWFQNSSVILFLNKKDLLEEKIMYSHLVDYFPEYDGPQRDAQAAREFILKMFVDLNPDSDKIIYSHFTCATDTENIRFVFAAVKDTILQLNLKEYNLV",
# GBG4_HUMAN
"MKEGMSNNSTTSISQARKAVEQLKMEACMDRVKVSQAAADLLAYCEAHVREDPLIIPVPASENPFREKKFFCTIL",
# GDIB_HUMAN
"MNEEYDVIVLGTGLTECILSGIMSVNGKKVLHMDRNPYYGGESASITPLEDLYKRFKIPGSPPESMGRGRDWNVDLIPKFLMANGQLVKMLLYTEVTRYLDFKVTEGSFVYKGGKIYKVPSTEAEALASSLMGLFEKRRFRKFLVYVANFDEKDPRTFEGIDPKKTTMRDVYKKFDLGQDVIDFTGHALALYRTDDYLDQPCYETINRIKLYSESLARYGKSPYLYPLYGLGELPQGFARLSAIYGGTYMLNKPIEEIIVQNGKVIGVKSEGEIARCKQLICDPSYVKDRVEKVGQVIRVICILSHPIKNTNDANSCQIIIPQNQVNRKSDIYVCMISFAHNVAAQGKYIAIVSTTVETKEPEKEIRPALELLEPIEQKFVSISDLLVPKDLGTESQIFISRTYDATTHFETTCDDIKNIYKRMTGSEFDFEEMKRKKNDIYGED",
# PDLI4_HUMAN
"MPHSVTLRGPSPWGFRLVGGRDFSAPLTISRVHAGSKAALAALCPGDLIQAINGESTELMTHLEAQNRIKGCHDHLTLSVSRPEGRSWPSAPDDSKAQAHRIHIDPEIQDGSPTTSRRPSGTGTGPEDGRPSLGSPYGQPPRFPVPHNGSSEATLPAQMSTLHVSPPPSADPARGLPRSRDCRVDLGSEVYRMLREPAEPVAAEPKQSGSFRYLQGMLEAGEGGDWPGPGGPRNLKPTASKLGAPLSGLQGLPECTRCGHGIVGTIVKARDKLYHPECFMCSDCGLNLKQRGYFFLDERLYCESHAKARVKPPEGYDVVAVYPNAKVELV",
# VASP_HUMAN
"MSETVICSSRATVMLYDDGNKRWLPAGTGPQAFSRVQIYHNPTANSFRVVGRKMQPDQQVVINCAIVRGVKYNQATPNFHQWRDARQVWGLNFGSKEDAAQFAAGMASALEALEGGGPPPPPALPTWSVPNGPSPEEVEQQKRQQPGPSEHIERRVSNAGGPPAPPAGGPPPPPGPPPPPGPPPPPGLPPSGVPAAAHGAGGGPPPAPPLPAAQGPGGGGAGAPGLAAAIAGAKLRKVSKQEEASGGPTAPKAESGRSGGGGLMEEMNAMLARRRKATQVGEKTPKDESANQEEPEARVPAQSESVRRPWEKNSTTLPRMKSSSSVTTSETQPCTPSSSDYSDLQRVKQELLEEVKKELQKVKEEIIEAFVQELRKRGSP",
# DYN2_HUMAN
"MGNRGMEELIPLVNKLQDAFSSIGQSCHLDLPQIAVVGGQSAGKSSVLENFVGRDFLPRGSGIVTRRPLILQLIFSKTEHAEFLHCKSKKFTDFDEVRQEIEAETDRVTGTNKGISPVPINLRVYSPHVLNLTLIDLPGITKVPVGDQPPDIEYQIKDMILQFISRESSLILAVTPANMDLANSDALKLAKEVDPQGLRTIGVITKLDLMDEGTDARDVLENKLLPLRRGYIGVVNRSQKDIEGKKDIRAALAAERKFFLSHPAYRHMADRMGTPHLQKTLNQQLTNHIRESLPALRSKLQSQLLSLEKEVEEYKNFRPDDPTRKTKALLQMVQQFGVDFEKRIEGSGDQVDTLELSGGARINRIFHERFPFELVKMEFDEKDLRREISYAIKNIHGVRTGLFTPDLAFEAIVKKQVVKLKEPCLKCVDLVIQELINTVRQCTSKLSSYPRLREETERIVTTYIREREGRTKDQILLLIDIEQSYINTNHEDFIGFANAQQRSTQLNKKRAIPNQGEILVIRRGWLTINNISLMKGGSKEYWFVLTAESLSWYKDEEEKEKKYMLPLDNLKIRDVEKGFMSNKHVFAIFNTEQRNVYKDLRQIELACDSQEDVDSWKASFLRAGVYPEKDQAENEDGAQENTFSMDPQLERQVETIRNLVDSYVAIINKSIRDLMPKTIMHLMINNTKAFIHHELLAYLYSSADQSSLMEESADQAQRRDDMLRMYHALKEALNIIGDISTSTVSTPVPPPVDDTWLQSASSHSPTPQRRPVSSIHPPGRPPAVRGPTPGPPLIPVPVGAAASFSAPPIPSRPGPQSVFANSDLFPAPPQIPSRPVRIPPGIPPGVPSRRPPAAPSRPTIIRPAEPSLLD",
# PPT1_HUMAN
"MASPGCLWLLAVALLPWTCASRALQHLDPPAPLPLVIWHGMGDSCCNPLSMGAIKKMVEKKIPGIYVLSLEIGKTLMEDVENSFFLNVNSQVTTVCQALAKDPKLQQGYNAMGFSQGGQFLRAVAQRCPSPPMINLISVGGQHQGVFGLPRCPGESSHICDFIRKTLNAGAYSKVVQERLVQAEYWHDPIKEDVYRNHSIFLADINQERGINESYKKNLMALKKFVMVKFLNDSIVDPVDSEWFGFYRSGQAKETIPLQETSLYTQDRLGLKEMDNAGQLVFLATEGDHLQLSEEWFYAHIIPFLG",
# RL14_HUMAN
"MVFRRFVEVGRVAYVSFGPHAGKLVAIVDVIDQNRALVDGPCTQVRRQAMPFKCMQLTDFILKFPHSAHQKYVRQAWQKADINTKWAATRWAKKIEARERKAKMTDFDRFKVMKAKKMRNRIIKNEVKKLQKAALLKASPKKAPGTKGTAAAAAAAAAAKVPAKKITAASKKAPAQKVPAQKATGQKAAPAPKAQKGQKAPAQKAPAPKASGKKA",
# FXR1_HUMAN
"MAELTVEVRGSNGAFYKGFIKDVHEDSLTVVFENNWQPERQVPFNEVRLPPPPDIKKEISEGDEVEVYSRANDQEPCGWWLAKVRMMKGEFYVIEYAACDATYNEIVTFERLRPVNQNKTVKKNTFFKCTVDVPEDLREACANENAHKDFKKAVGACRIFYHPETTQLMILSASEATVKRVNILSDMHLRSIRTKLMLMSRNEEATKHLECTKQLAAAFHEEFVVREDLMGLAIGTHGSNIQQARKVPGVTAIELDEDTGTFRIYGESADAVKKARGFLEFVEDFIQVPRNLVGKVIGKNGKVIQEIVDKSGVVRVRIEGDNENKLPREDGMVPFVFVGTKESIGNVQVLLEYHIAYLKEVEQLRMERLQIDEQLRQIGSRSYSGRGRGRRGPNYTSGYGTNSELSNPSETESERKDELSDWSLAGEDDRDSRHQRDSRRRPGGRGRSVSGGRGRGGPRGGKSSISSVLKDPDSNPYSLLDNTESDQTADTDASESHHSTNRRRRSRRRRTDEDAVLMDGMTESDTASVNENGLVTVADYISRAESQSRQRNLPRETLAKNKKEMAKDVIEEHGPSEKAINGPTSASGDDISKLQRTPGEEKINTLKEENTQEAAVLNGVS",
# FXR2_HUMAN
"MGGLASGGDVEPGLPVEVRGSNGAFYKGFVKDVHEDSVTIFFENNWQSERQIPFGDVRLPPPADYNKEITEGDEVEVYSRANEQEPCGWWLARVRMMKGDFYVIEYAACDATYNEIVTLERLRPVNPNPLATKGSFFKVTMAVPEDLREACSNENVHKEFKKALGANCIFLNITNSELFILSTTEAPVKRASLLGDMHFRSLRTKLLLMSRNEEATKHLETSKQLAAAFQEEFTVREDLMGLAIGTHGANIQQARKVPGVTAIELGEETCTFRIYGETPEACRQARSYLEFSEDSVQVPRNLVGKVIGKNGKVIQEIVDKSGVVRVRVEGDNDKKNPREEGMVPFIFVGTRENISNAQALLEYHLSYLQEVEQLRLERLQIDEQLRQIGLGFRPPGSGRGSGGSDKAGYSTDESSSSSLHATRTYGGSYGGRGRGRRTGGPAYGPSSDVSTASETESEKREEPNRAGPGDRDPPTRGEESRRRPTGGRGRGPPPAPRPTSRYNSSSISSVLKDPDSNPYSLLDTSEPEPPVDSEPGEPPPASARRRRSRRRRTDEDRTVMDGGLESDGPNMTENGLEDESRPQRRNRSRRRRNRGNRTDGSISGDRQPVTVADYISRAESQSRQRPPLERTKPSEDSLSGQKGDSVSKLPKGPSENGELSAPLELGSMVNGVS",
# RAB5C_HUMAN
"MAGRGGAARPNGPAAGNKICQFKLVLLGESAVGKSSLVLRFVKGQFHEYQESTIGAAFLTQTVCLDDTTVKFEIWDTAGQERYHSLAPMYYRGAQAAIVVYDITNTDTFARAKNWVKELQRQASPNIVIALAGNKADLASKRAVEFQEAQAYADDNSLLFMETSAKTAMNVNEIFMAIAKKLPKNEPQNATGAPGRNRGVDLQENNPASRSQCCSN",
# RAB7A_HUMAN
"MTSRKKVLLKVIILGDSGVGKTSLMNQYVNKKFSNQYKATIGADFLTKEVMVDDRLVTMQIWDTAGQERFQSLGVAFYRGADCCVLVFDVTAPNTFKTLDSWRDEFLIQASPRDPENFPFVVLGNKIDLENRQVATKRAQAWCYSKNNIPYFETSAKEAINVEQAFQTIARNALKQETEVELYNEFPEPIKLDKNDRAKASAESCSC",
# P2RX1_HUMAN
"MARRFQEELAAFLFEYDTPRMVLVRNKKVGVIFRLIQLVVLVYVIGWVFLYEKGYQTSSGLISSVSVKLKGLAVTQLPGLGPQVWDVADYVFPAQGDNSFVVMTNFIVTPKQTQGYCAEHPEGGICKEDSGCTPGKAKRKAQGIRTGKCVAFNDTVKTCEIFGWCPVEVDDDIPRPALLREAENFTLFIKNSISFPRFKVNRRNLVEEVNAAHMKTCLFHKTLHPLCPVFQLGYVVQESGQNFSTLAEKGGVVGITIDWHCDLDWHVRHCRPIYEFHGLYEEKNLSPGFNFRFARHFVENGTNYRHLFKVFGIRFDILVDGKAGKFDIIPTMTTIGSGIGIFGVATVLCDLLLLHILPKRHYYKQKKFKYAEDMGPGAAERDLAATSSTLGLQENMRTS",
# P2RY4_HUMAN
"MASTESSLLRSLGLSPGPGSSEVELDCWFDEDFKFILLPVSYAVVFVLGLGLNAPTLWLFIFRLRPWDATATYMFHLALSDTLYVLSLPTLIYYYAAHNHWPFGTEICKFVRFLFYWNLYCSVLFLTCISVHRYLGICHPLRALRWGRPRLAGLLCLAVWLVVAGCLVPNLFFVTTSNKGTTVLCHDTTRPEEFDHYVHFSSAVMGLLFGVPCLVTLVCYGLMARRLYQPLPGSAQSSSRLRSLRTIAVVLTVFAVCFVPFHITRTIYYLARLLEADCRVLNIVNVVYKVTRPLASANSCLDPVLYLLTGDKYRRQLRQLCGGGKPQPRTAASSLALVSLPEDSSCRWAATPQDSSCSTPRADRL",
# MECP2_HUMAN
"MVAGMLGLREEKSEDQDLQGLKDKPLKFKKVKKDKKEEKEGKHEPVQPSAHHSAEPAEAGKAETSEGSGSAPAVPEASASPKQRRSIIRDRGPMYDDPTLPEGWTRKLKQRKSGRSAGKYDVYLINPQGKAFRSKVELIAYFEKVGDTSLDPNDFDFTVTGRGSPSRREQKPPKKPKSPKAPGTGRGRGRPKGSGTTRPKAATSEGVQVKRVLEKSPGKLLVKMPFQTSPGGKAEGGGATTSTQVMVIKRPGRKRKAEADPQAIPKKRGRKPGSVVAAAAAEAKKKAVKESSIRSVQETVLPIKKRKTRETVSIEVKEVVKPLLVSTLGEKSGKGLKTCKSPGRKSKESSPKGRSSSASSPPKKEHHHHHHHSESPKAPVPLLPPLPPPPPEPESSEDPTSPPEPQDLSSSVCKEEKMPRGGSLESDGCPKEPAKTQPAVATAATAAEKYKHRGEGERKDIVSSSMPRPNREEPVDSRTPVTERVS",
# GPM6A_HUMAN
"MEENMEEGQTQKGCFECCIKCLGGIPYASLIATILLYAGVALFCGCGHEALSGTVNILQTYFEMARTAGDTLDVFTMIDIFKYVIYGIAAAFFVYGILLMVEGFFTTGAIKDLYGDFKITTCGRCVSAWFIMLTYLFMLAWLGVTAFTSLPVYMYFNLWTICRNTTLVEGANLCLDLRQFGIVTIGEEKKICTVSENFLRMCESTELNMTFHLFIVALAGAGAAVIAMVHYLMVLSANWAYVKDACRMQKYEDIKSKEEQELHDIHSTRSKERLNAYT",
# CLCN3_HUMAN
"MESEQLFHRGYYRNSYNSITSASSDEELLDGAGVIMDFQTSEDDNLLDGDTAVGTHYTMTNGGSINSSTHLLDLLDEPIPGVGTYDDFHTIDWVREKCKDRERHRRINSKKKESAWEMTKSLYDAWSGWLVVTLTGLASGALAGLIDIAADWMTDLKEGICLSALWYNHEQCCWGSNETTFEERDKCPQWKTWAELIIGQAEGPGSYIMNYIMYIFWALSFAFLAVSLVKVFAPYACGSGIPEIKTILSGFIIRGYLGKWTLMIKTITLVLAVASGLSLGKEGPLVHVACCCGNIFSYLFPKYSTNEAKKREVLSAASAAGVSVAFGAPIGGVLFSLEEVSYYFPLKTLWRSFFAALVAAFVLRSINPFGNSRLVLFYVEYHTPWYLFELFPFILLGVFGGLWGAFFIRANIAWCRRRKSTKFGKYPVLEVIIVAAITAVIAFPNPYTRLNTSELIKELFTDCGPLESSSLCDYRNDMNASKIVDDIPDRPAGIGVYSAIWQLCLALIFKIIMTVFTFGIKVPSGLFIPSMAIGAIAGRIVGIAVEQLAYYHHDWFIFKEWCEVGADCITPGLYAMVGAAACLGGVTRMTVSLVVIVFELTGGLEYIVPLMAAVMTSKWVGDAFGREGIYEAHIRLNGYPFLDAKEEFTHTTLAADVMRPRRNDPPLAVLTQDNMTVDDIENMINETSYNGFPVIMSKESQRLVGFALRRDLTIAIESARKKQEGIVGSSRVCFAQHTPSLPAESPRPLKLRSILDMSPFTVTDHTPMEIVVDIFRKLGLRQCLVTHNGRLLGIITKKDILRHMAQTANQDPASIMFN",
# VAMP7_HUMAN
"MAILFAVVARGTTILAKHAWCGGNFLEVTEQILAKIPSENNKLTYSHGNYLFHYICQDRIVYLCITDDDFERSRAFNFLNEIKKRFQTTYGSRAQTALPYAMNSEFSSVLAAQLKHHSENKGLDKVMETQAQVDELKGIMVRNIDLVAQRGERLELLIDKTENLVDSSVTFKTTSRNLARAMCMKNLKLTIIIIIVSIVFIYIIVSPLCGGFTWPSCVKK",
# ROA3_HUMAN
"MEVKPPPGRPQPDSGRRRRRRGEEGHDPKEPEQLRKLFIGGLSFETTDDSLREHFEKWGTLTDCVVMRDPQTKRSRGFGFVTYSCVEEVDAAMCARPHKVDGRVVEPKRAVSREDSVKPGAHLTVKKIFVGGIKEDTEEYNLRDYFEKYGKIETIEVMEDRQSGKKRGFAFVTFDDHDTVDKIVVQKYHTINGHNCEVKKALSKQEMQSAGSQRGRGGGSGNFMGRGGNFGGGGGNFGRGGNFGGRGGYGGGGGGSRGSYGGGDGGYNGFGGDGGNYGGGPGYSSRGGYGGGGPGYGNQGGGYGGGGGYDGYNEGGNFGGGNYGGGGNYNDFGNYSGQQQSNYGPMKGGSFGGRSSGSPYGGGYGSGGGSGGYGSRRF",
# HNRPM_HUMAN
"MAAGVEAAAEVAATEIKMEEESGAPGVPSGNGAPGPKGEGERPAQNEKRKEKNIKRGGNRFEPYANPTKRYRAFITNIPFDVKWQSLKDLVKEKVGEVTYVELLMDAEGKSRGCAVVEFKMEESMKKAAEVLNKHSLSGRPLKVKEDPDGEHARRAMQKVMATTGGMGMGPGGPGMITIPPSILNNPNIPNEIIHALQAGRLGSTVFVANLDYKVGWKKLKEVFSMAGVVVRADILEDKDGKSRGIGTVTFEQSIEAVQAISMFNGQLLFDRPMHVKMDERALPKGDFFPPERPQQLPHGLGGIGMGLGPGGQPIDANHLNKGIGMGNIGPAGMGMEGIGFGINKMGGMEGPFGGGMENMGRFGSGMNMGRINEILSNALKRGEIIAKQGGGGGGGSVPGIERMGPGIDRLGGAGMERMGAGLGHGMDRVGSEIERMGLVMDRMGSVERMGSGIERMGPLGLDHMASSIERMGQTMERIGSGVERMGAGMGFGLERMAAPIDRVGQTIERMGSGVERMGPAIERMGLSMERMVPAGMGAGLERMGPVMDRMATGLERMGANNLERMGLERMGANSLERMGLERMGANSLERMGPAMGPALGAGIERMGLAMGGGGGASFDRAIEMERGNFGGSFAGSFGGAGGHAPGVARKACQIFVRNLPFDFTWKMLKDKFNECGHVLYADIKMENGKSKGCGVVKFESPEVAERACRMMNGMKLSGREIDVRIDRNA",
# IMA1_HUMAN
"MSTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQMLKRRNVSSFPDDATSPLQENRNNQGTVNWSVDDIVKGINSSNVENQLQATQAARKLLSREKQPPIDNIIRAGLIPKFVSFLGRTDCSPIQFESAWALTNIASGTSEQTKAVVDGGAIPAFISLLASPHAHISEQAVWALGNIAGDGSVFRDLVIKYGAVDPLLALLAVPDMSSLACGYLRNLTWTLSNLCRNKNPAPPIDAVEQILPTLVRLLHHDDPEVLADTCWAISYLTDGPNERIGMVVKTGVVPQLVKLLGASELPIVTPALRAIGNIVTGTDEQTQVVIDAGALAVFPSLLTNPKTNIQKEATWTMSNITAGRQDQIQQVVNHGLVPFLVSVLSKADFKTQKEAVWAVTNYTSGGTVEQIVYLVHCGIIEPLMNLLTAKDTKIILVILDAISNIFQAAEKLGETEKLSIMIEECGGLDKIEALQNHENESVYKASLSLIEKYFSVEEEEDQNVVPETTSEGYTFQVQDGAPGTFNF",
# IMA5_HUMAN
"MTTPGKENFRLKSYKNKSLNPDEMRRRREEEGLQLRKQKREEQLFKRRNVATAEEETEEEVMSDGGFHEAQISNMEMAPGGVITSDMIEMIFSKSPEQQLSATQKFRKLLSKEPNPPIDEVISTPGVVARFVEFLKRKENCTLQFESAWVLTNIASGNSLQTRIVIQAGAVPIFIELLSSEFEDVQEQAVWALGNIAGDSTMCRDYVLDCNILPPLLQLFSKQNRLTMTRNAVWALSNLCRGKSPPPEFAKVSPCLNVLSWLLFVSDTDVLADACWALSYLSDGPNDKIQAVIDAGVCRRLVELLMHNDYKVVSPALRAVGNIVTGDDIQTQVILNCSALQSLLHLLSSPKESIKKEACWTISNITAGNRAQIQTVIDANIFPALISILQTAEFRTRKEAAWAITNATSGGSAEQIKYLVELGCIKPLCDLLTVMDSKIVQVALNGLENILRLGEQEAKRNGTGINPYCALIEEAYGLDKIEFLQSHENQEIYQKAFDLIEHYFGTEDEDSSIAPQVDLNQQQYIFQQCEAPMEGFQL",
# DGKE_HUMAN
"MEAERRPAPGSPSEGLFADGHLILWTLCSVLLPVFITFWCSLQRSRRQLHRRDIFRKSKHGWRDTDLFSQPTYCCVCAQHILQGAFCDCCGLRVDEGCLRKADKRFQCKEIMLKNDTKVLDAMPHHWIRGNVPLCSYCMVCKQQCGCQPKLCDYRCIWCQKTVHDECMKNSLKNEKCDFGEFKNLIIPPSYLTSINQMRKDKKTDYEVLASKLGKQWTPLIILANSRSGTNMGEGLLGEFRILLNPVQVFDVTKTPPIKALQLCTLLPYYSARVLVCGGDGTVGWVLDAVDDMKIKGQEKYIPQVAVLPLGTGNDLSNTLGWGTGYAGEIPVAQVLRNVMEADGIKLDRWKVQVTNKGYYNLRKPKEFTMNNYFSVGPDALMALNFHAHREKAPSLFSSRILNKAVYLFYGTKDCLVQECKDLNKKVELELDGERVALPSLEGIIVLNIGYWGGGCRLWEGMGDETYPLARHDDGLLEVVGVYGSFHCAQIQVKLANPFRIGQAHTVRLILKCSMMPMQVDGEPWAQGPCTVTITHKTHAMMLYFSGEQTDDDISSTSDQEDIKATE",
# GDIR1_HUMAN
"MAEQEPTAEQLAQIAAENEEDEHSVNYKPPAQKSIQEIQELDKDDESLRKYKEALLGRVAVSADPNVPNVVVTGLTLVCSSAPGPLELDLTGDLESFKKQSFVLKEGVEYRIKISFRVNREIVSGMKYIQHTYRKGVKIDKTDYMVGSYGPRAEEYEFLTPVEEAPKGMLARGSYSIKSRFTDDDKTDHLSWEWNLTIKKDWKD",
# HNRPF_HUMAN
"MMLGPEGGEGFVVKLRGLPWSCSVEDVQNFLSDCTIHDGAAGVHFIYTREGRQSGEAFVELGSEDDVKMALKKDRESMGHRYIEVFKSHRTEMDWVLKHSGPNSADSANDGFVRLRGLPFGCTKEEIVQFFSGLEIVPNGITLPVDPEGKITGEAFVQFASQELAEKALGKHKERIGHRYIEVFKSSQEEVRSYSDPPLKFMSVQRPGPYDRPGTARRYIGIVKQAGLERMRPGAYSTGYGGYEEYSGLSDGYGFTTDLFGRDLSYCLSGMYDHRYGDSEFTVQSTTGHCVHMRGLPYKATENDIYNFFSPLNPVRVHIEIGPDGRVTGEADVEFATHEEAVAAMSKDRANMQHRYIELFLNSTTGASNGAYSSQVMQGMGVSAAQATYSGLESQSVSGCYGAGYSGQNSMGGYD",
# EFNB2_HUMAN
"MAVRRDSVWKYCWGVLMVLCRTAISKSIVLEPIYWNSSNSKFLPGQGLVLYPQIGDKLDIICPKVDSKTVGQYEYYKVYMVDKDQADRCTIKKENTPLLNCAKPDQDIKFTIKFQEFSPNLWGLEFQKNKDYYIISTSNGSLEGLDNQEGGVCQTRAMKILMKVGQDASSAGSTRNKDPTRRPELEAGTNGRSSTTSPFVKPNPGSSTDGNSAGHSGNNILGSEVALFAGIASGCIIFIVIIITLVVLLLKYRRRHRKHSPQHTTTLSLSTLATPKRSGNNNGSEPSDIIIPLRTADSVFCPHYEKVSGDYGHPVYIVQEMPPQSPANIYYKV",
# EFNA5_HUMAN
"MLHVEMLTLVFLVLWMCVFSQDPGSKAVADRYAVYWNSSNPRFQRGDYHIDVCINDYLDVFCPHYEDSVPEDKTERYVLYMVNFDGYSACDHTSKGFKRWECNRPHSPNGPLKFSEKFQLFTPFSLGFEFRPGREYFYISSAIPDNGRRSCLKLKVFVRPTNSCMKTIGVHDRVFDVNDKVENSLEPADDTVHESAEPSRGENAAQTPRIPSRLLAILLFLLAMLLTL",
# DGKQ_HUMAN
"MAAAAEPGARAWLGGGSPRPGSPACSPVLGSGGRARPGPGPGPGPERAGVRAPGPAAAPGHSFRKVTLTKPTFCHLCSDFIWGLAGFLCDVCNFMSHEKCLKHVRIPCTSVAPSLVRVPVAHCFGPRGLHKRKFCAVCRKVLEAPALHCEVCELHLHPDCVPFACSDCRQCHQDGHQDHDTHHHHWREGNLPSGARCEVCRKTCGSSDVLAGVRCEWCGVQAHSLCSAALAPECGFGRLRSLVLPPACVRLLPGGFSKTQSFRIVEAAEPGEGGDGADGSAAVGPGRETQATPESGKQTLKIFDGDDAVRRSQFRLVTVSRLAGAEEVLEAALRAHHIPEDPGHLELCRLPPSSQACDAWAGGKAGSAVISEEGRSPGSGEATPEAWVIRALPRAQEVLKIYPGWLKVGVAYVSVRVTPKSTARSVVLEVLPLLGRQAESPESFQLVEVAMGCRHVQRTMLMDEQPLLDRLQDIRQMSVRQVSQTRFYVAESRDVAPHVSLFVGGLPPGLSPEEYSSLLHEAGATKATVVSVSHIYSSQGAVVLDVACFAEAERLYMLLKDMAVRGRLLTALVLPDLLHAKLPPDSCPLLVFVNPKSGGLKGRDLLCSFRKLLNPHQVFDLTNGGPLPGLHLFSQVPCFRVLVCGGDGTVGWVLGALEETRYRLACPEPSVAILPLGTGNDLGRVLRWGAGYSGEDPFSVLLSVDEADAVLMDRWTILLDAHEAGSAENDTADAEPPKIVQMSNYCGIGIDAELSLDFHQAREEEPGKFTSRLHNKGVYVRVGLQKISHSRSLHKQIRLQVERQEVELPSIEGLIFINIPSWGSGADLWGSDSDTRFEKPRMDDGLLEVVGVTGVVHMGQVQGGLRSGIRIAQGSYFRVTLLKATPVQVDGEPWVQAPGHMIISAAGPKVHMLRKAKQKPRRAGTTRDARADAAPAPESDPR",
# DAPK1_HUMAN
"MTVFRQENVDDYYDTGEELGSGQFAVVKKCREKSTGLQYAAKFIKKRRTKSSRRGVSREDIEREVSILKEIQHPNVITLHEVYENKTDVILILELVAGGELFDFLAEKESLTEEEATEFLKQILNGVYYLHSLQIAHFDLKPENIMLLDRNVPKPRIKIIDFGLAHKIDFGNEFKNIFGTPEFVAPEIVNYEPLGLEADMWSIGVITYILLSGASPFLGDTKQETLANVSAVNYEFEDEYFSNTSALAKDFIRRLLVKDPKKRMTIQDSLQHPWIKPKDTQQALSRKASAVNMEKFKKFAARKKWKQSVRLISLCQRLSRSFLSRSNMSVARSDDTLDEEDSFVMKAIIHAINDDNVPGLQHLLGSLSNYDVNQPNKHGTPPLLIAAGCGNIQILQLLIKRGSRIDVQDKGGSNAVYWAARHGHVDTLKFLSENKCPLDVKDKSGEMALHVAARYGHADVAQLLCSFGSNPNIQDKEEETPLHCAAWHGYYSVAKALCEAGCNVNIKNREGETPLLTASARGYHDIVECLAEHGADLNACDKDGHIALHLAVRRCQMEVIKTLLSQGCFVDYQDRHGNTPLHVACKDGNMPIVVALCEANCNLDISNKYGRTPLHLAANNGILDVVRYLCLMGASVEALTTDGKTAEDLARSEQHEHVAGLLARLRKDTHRGLFIQQLRPTQNLQPRIKLKLFGHSGSGKTTLVESLKCGLLRSFFRRRRPRLSSTNSSRFPPSPLASKPTVSVSINNLYPGCENVSVRSRSMMFEPGLTKGMLEVFVAPTHHPHCSADDQSTKAIDIQNAYLNGVGDFSVWEFSGNPVYFCCYDYFAANDPTSIHVVVFSLEEPYEIQLNQVIFWLSFLKSLVPVEEPIAFGGKLKNPLQVVLVATHADIMNVPRPAGGEFGYDKDTSLLKEIRNRFGNDLHISNKLFVLDAGASGSKDMKVLRNHLQEIRSQIVSVCPPMTHLCEKIISTLPSWRKLNGPNQLMSLQQFVYDVQDQLNPLASEEDLRRIAQQLHSTGEINIMQSETVQDVLLLDPRWLCTNVLGKLLSVETPRALHHYRGRYTVEDIQRLVPDSDVEELLQILDAMDICARDLSSGTMVDVPALIKTDNLHRSWADEEDEVMVYGGVRIVPVEHLTPFPCGIFHKVQVNLCRWIHQQSTEGDADIRLWVNGCKLANRGAELLVLLVNHGQGIEVQVRGLETEKIKCCLLLDSVCSTIENVMATTLPGLLTVKHYLSPQQLREHHEPVMIYQPRDFFRAQTLKETSLTNTMGGYKESFSSIMCFGCHDVYSQASLGMDIHASDLNLLTRRKLSRLLDPPDPLGKDWCLLAMNLGLPDLVAKYNTSNGAPKDFLPSPLHALLREWTTYPESTVGTLMSKLRELGRRDAADFLLKASSVFKINLDGNGQEAYASSCNSGTSYNSISSVVSR",
# LIMK1_HUMAN
"MRLTLLCCTWREERMGEEGSELPVCASCGQRIYDGQYLQALNADWHADCFRCCDCSASLSHQYYEKDGQLFCKKDYWARYGESCHGCSEQITKGLVMVAGELKYHPECFICLTCGTFIGDGDTYTLVEHSKLYCGHCYYQTVVTPVIEQILPDSPGSHLPHTVTLVSIPASSHGKRGLSVSIDPPHGPPGCGTEHSHTVRVQGVDPGCMSPDVKNSIHVGDRILEINGTPIRNVPLDEIDLLIQETSRLLQLTLEHDPHDTLGHGLGPETSPLSSPAYTPSGEAGSSARQKPVLRSCSIDRSPGAGSLGSPASQRKDLGRSESLRVVCRPHRIFRPSDLIHGEVLGKGCFGQAIKVTHRETGEVMVMKELIRFDEETQRTFLKEVKVMRCLEHPNVLKFIGVLYKDKRLNFITEYIKGGTLRGIIKSMDSQYPWSQRVSFAKDIASGMAYLHSMNIIHRDLNSHNCLVRENKNVVVADFGLARLMVDEKTQPEGLRSLKKPDRKKRYTVVGNPYWMAPEMINGRSYDEKVDVFSFGIVLCEIIGRVNADPDYLPRTMDFGLNVRGFLDRYCPPNCPPSFFPITVRCCDLDPEKRPSFVKLEHWLETLRMHLAGHLPLGPQLEQLDRGFWETYRRGESGLPAHPEVPD",
# AP3M2_HUMAN
"MIHSLFLINSSGDIFLEKHWKSVVSRSVCDYFFEAQERATEAENVPPVIPTPHHYLLSVYRHKIFFVAVIQTEVPPLFVIEFLHRVVDTFQDYFGVCSEPVIKDNVVVVYEVLEEMLDNGFPLATESNILKELIKPPTILRTVVNTITGSTNVGDQLPTGQLSVVPWRRTGVKYTNNEAYFDVIEEIDAIIDKSGSTITAEIQGVIDACVKLTGMPDLTLSFMNPRLLDDVSFHPCVRFKRWESERILSFIPPDGNFRLLSYHVSAQNLVAIPVYVKHNISFRDSSSLGRFEITVGPKQTMGKTIEGVTVTSQMPKGVLNMSLTPSQGTHTFDPVTKMLSWDVGKINPQKLPSLKGTMSLQAGASKPDENPTINLQFKIQQLAISGLKVNRLDMYGEKYKPFKGIKYMTKAGKFQVRT",
# AP2S1_HUMAN
"MIRFILIQNRAGKTRLAKWYMQFDDDEKQKLIEEVHAVVTVRDAKHTNFVEFRNFKIIYRRYAGLYFCICVDVNDNNLAYLEAIHNFVEVLNEYFHNVCELDLVFNFYKVYTVVDEMFLAGEIRETSQTKVLKQLLMLQSLE",
# ITA8_HUMAN
"MSPGASRGPRGSQAPLIAPLCCAAAALGMLLWSPACQAFNLDVEKLTVYSGPKGSYFGYAVDFHIPDARTASVLVGAPKANTSQPDIVEGGAVYYCPWPAEGSAQCRQIPFDTTNNRKIRVNGTKEPIEFKSNQWFGATVKAHKGKVVACAPLYHWRTLKPTPEKDPVGTCYVAIQNFSAYAEFSPCRNSNADPEGQGYCQAGFSLDFYKNGDLIVGGPGSFYWQGQVITASVADIIANYSFKDILRKLAGEKQTEVAPASYDDSYLGYSVAAGEFTGDSQQELVAGIPRGAQNFGYVSIINSTDMTFIQNFTGEQMASYFGYTVVVSDVNSDGLDDVLVGAPLFMEREFESNPREVGQIYLYLQVSSLLFRDPQILTGTETFGRFGSAMAHLGDLNQDGYNDIAIGVPFAGKDQRGKVLIYNGNKDGLNTKPSQVLQGVWASHAVPSGFGFTLRGDSDIDKNDYPDLIVGAFGTGKVAVYRARPVVTVDAQLLLHPMIINLENKTCQVPDSMTSAACFSLRVCASVTGQSIANTIVLMAEVQLDSLKQKGAIKRTLFLDNHQAHRVFPLVIKRQKSHQCQDFIVYLRDETEFRDKLSPINISLNYSLDESTFKEGLEVKPILNYYRENIVSEQAHILVDCGEDNLCVPDLKLSARPDKHQVIIGDENHLMLIINARNEGEGAYEAELFVMIPEEADYVGIERNNKGFRPLSCEYKMENVTRMVVCDLGNPMVSGTNYSLGLRFAVPRLEKTNMSINFDLQIRSSNKDNPDSNFVSLQINITAVAQVEIRGVSHPPQIVLPIHNWEPEEEPHKEEEVGPLVEHIYELHNIGPSTISDTILEVGWPFSARDEFLLYIFHIQTLGPLQCQPNPNINPQDIKPAASPEDTPELSAFLRNSTIPHLVRKRDVHVVEFHRQSPAKILNCTNIECLQISCAVGRLEGGESAVLKVRSRLWAHTFLQRKNDPYALASLVSFEVKKMPYTDQPAKLPEGSIVIKTSVIWATPNVSFSIPLWVIILAILLGLLVLAILTLALWKCGFFDRARPPQEDMTDREQLTNDKTPEA",
# MK10_HUMAN
"MSLHFLYYCSEPTLDVKIAFCQGFDKQVDVSYIAKHYNMSKSKVDNQFYSVEVGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDAVLDRNVAIKKLSRPFQNQTHAKRAYRELVLMKCVNHKNIISLLNVFTPQKTLEEFQDVYLVMELMDANLCQVIQMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTAGTSFMMTPYVVTRYYRAPEVILGMGYKENVDIWSVGCIMGEMVRHKILFPGRDYIDQWNKVIEQLGTPCPEFMKKLQPTVRNYVENRPKYAGLTFPKLFPDSLFPADSEHNKLKASQARDLLSKMLVIDPAKRISVDDALQHPYINVWYDPAEVEAPPPQIYDKQLDEREHTIEEWKELIYKEVMNSEEKTKNGVVKGQPSPSGAAVNSSESLPPSSSVNDISSMSTDQTLASDTDSSLEASAGPLGCCR",
# MOT1_HUMAN
"MPPAVGGPVGYTPPDGGWGWAVVIGAFISIGFSYAFPKSITVFFKEIEGIFHATTSEVSWISSIMLAVMYGGGPISSILVNKYGSRIVMIVGGCLSGCGLIAASFCNTVQQLYVCIGVIGGLGLAFNLNPALTMIGKYFYKRRPLANGLAMAGSPVFLCTLAPLNQVFFGIFGWRGSFLILGGLLLNCCVAGALMRPIGPKPTKAGKDKSKASLEKAGKSGVKKDLHDANTDLIGRHPKQEKRSVFQTINQFLDLTLFTHRGFLLYLSGNVIMFFGLFAPLVFLSSYGKSQHYSSEKSAFLLSILAFVDMVARPSMGLVANTKPIRPRIQYFFAASVVANGVCHMLAPLSTTYVGFCVYAGFFGFAFGWLSSVLFETLMDLVGPQRFSSAVGLVTIVECCPVLLGPPLLGRLNDMYGDYKYTYWACGVVLIISGIYLFIGMGINYRLLAKEQKANEQKKESKEEETSIDVAGKPNEVTKAAESPDQKDTDGGPKEEESPV",
# VMAT1_HUMAN
"MLRTILDAPQRLLKEGRASRQLVLVVVFVALLLDNMLFTVVVPIVPTFLYDMEFKEVNSSLHLGHAGSSPHALASPAFSTIFSFFNNNTVAVEESVPSGIAWMNDTASTIPPPATEAISAHKNNCLQGTGFLEEEITRVGVLFASKAVMQLLVNPFVGPLTNRIGYHIPMFAGFVIMFLSTVMFAFSGTYTLLFVARTLQGIGSSFSSVAGLGMLASVYTDDHERGRAMGTALGGLALGLLVGAPFGSVMYEFVGKSAPFLILAFLALLDGALQLCILQPSKVSPESAKGTPLFMLLKDPYILVAAGSICFANMGVAILEPTLPIWMMQTMCSPKWQLGLAFLPASVSYLIGTNLFGVLANKMGRWLCSLIGMLVVGTSLLCVPLAHNIFGLIGPNAGLGLAIGMVDSSMMPIMGHLVDLRHTSVYGSVYAIADVAFCMGFAIGPSTGGAIVKAIGFPWLMVITGVINIVYAPLCYYLRSPPAKEEKLAILSQDCPMETRMYATQKPTKEFPLGEDSDEEPDHEE",
# HAP1_HUMAN
"MRPKRLGRCCAGSRLGPGDPAALTCAPSPSASPAPEPSAQPQARGTGQRVGSRATSGSQFLSEARTGARPASEAGAKAGARRPSAFSAIQGDVRSMPDNSDAPWTRFVFQGPFGSRATGRGTGKAAGIWKTPAAYVGRRPGVSGPERAAFIRELEEALCPNLPPPVKKITQEDVKVMLYLLEELLPPVWESVTYGMVLQRERDLNTAARIGQSLVKQNSVLMEENSKLEALLGSAKEEILYLRHQVNLRDELLQLYSDSDEEDEDEEEEEEEKEAEEEQEEEEAEEDLQCAHPCDAPKLISQEALLHQHHCPQLEALQEKLRLLEEENHQLREEASQLDTLEDEEQMLILECVEQFSEASQQMAELSEVLVLRLENYERQQQEVARLQAQVLKLQQRCRMYGAETEKLQKQLASEKEIQMQLQEESVWVGSQLQDLREKYMDCGGMLIEMQEEVKTLRQQPPVSTGSATHYPYSVPLETLPGFQETLAEELRTSLRRMISDPVYFMERNYEMPRGDTSSLRYDFRYSEDREQVRGFEAEEGLMLAADIMRGEDFTPAEEFVPQEELGAAKKVPAEEGVMEEAELVSEETEGWEEVELELDEATRMNVVTSALEASGLGPSHLDMNYVLQQLANWQDAHYRRQLRWKMLQKGECPHGALPAASRTSCRSSCR",
# CA2D1_HUMAN
"MAAGCLLALTLTLFQSLLIGPSSEEPFPSAVTIKSWVDKMQEDLVTLAKTASGVNQLVDIYEKYQDLYTVEPNNARQLVEIAARDIEKLLSNRSKALVRLALEAEKVQAAHQWREDFASNEVVYYNAKDDLDPEKNDSEPGSQRIKPVFIEDANFGRQISYQHAAVHIPTDIYEGSTIVLNELNWTSALDEVFKKNREEDPSLLWQVFGSATGLARYYPASPWVDNSRTPNKIDLYDVRRRPWYIQGAASPKDMLILVDVSGSVSGLTLKLIRTSVSEMLETLSDDDFVNVASFNSNAQDVSCFQHLVQANVRNKKVLKDAVNNITAKGITDYKKGFSFAFEQLLNYNVSRANCNKIIMLFTDGGEERAQEIFNKYNKDKKVRVFTFSVGQHNYDRGPIQWMACENKGYYYEIPSIGAIRINTQEYLDVLGRPMVLAGDKAKQVQWTNVYLDALELGLVITGTLPVFNITGQFENKTNLKNQLILGVMGVDVSLEDIKRLTPRFTLCPNGYYFAIDPNGYVLLHPNLQPKPIGVGIPTINLRKRRPNIQNPKSQEPVTLDFLDAELENDIKVEIRNKMIDGESGEKTFRTLVKSQDERYIDKGNRTYTWTPVNGTDYSLALVLPTYSFYYIKAKLEETITQARYSETLKPDNFEESGYTFIAPRDYCNDLKISDNNTEFLLNFNEFIDRKTPNNPSCNADLINRVLLDAGFTNELVQNYWSKQKNIKGVKARFVVTDGGITRVYPKEAGENWQENPETYEDSFYKRSLDNDNYVFTAPYFNKSGPGAYESGIMVSKAVEIYIQGKLLKPAVVGIKIDVNSWIENFTKTSIRDPCAGPVCDCKRNSDVMDCVILDDGGFLLMANHDDYTNQIGRFFGEIDPSLMRHLVNISVYAFNKSYDYQSVCEPGAAPKQGAGHRSAYVPSVADILQIGWWATAAAWSILQQFLLSLTFPRLLEAVEMEDDDFTASLSKQSCITEQTQYFFDNDSKSFSGVLDCGNCSRIFHGEKLMNTNLIFIMVESKGTCPCDTRLLIQAEQTSDGPNPCDMVKQPRYRKGPDVCFDNNVLEDYTDCGGVSGLNPSLWYIIGIQFLLLWLVSGSTHRLL",
# UBP14_HUMAN
"MPLYSVTVKWGKEKFEGVELNTDEPPMVFKAQLFALTGVQPARQKVMVKGGTLKDDDWGNIKIKNGMTLLMMGSADALPEEPSAKTVFVEDMTEEQLASAMELPCGLTNLGNTCYMNATVQCIRSVPELKDALKRYAGALRASGEMASAQYITAALRDLFDSMDKTSSSIPPIILLQFLHMAFPQFAEKGEQGQYLQQDANECWIQMMRVLQQKLEAIEDDSVKETDSSSASAATPSKKKSLIDQFFGVEFETTMKCTESEEEEVTKGKENQLQLSCFINQEVKYLFTGLKLRLQEEITKQSPTLQRNALYIKSSKISRLPAYLTIQMVRFFYKEKESVNAKVLKDVKFPLMLDMYELCTPELQEKMVSFRSKFKDLEDKKVNQQPNTSDKKSSPQKEVKYEPFSFADDIGSNNCGYYDLQAVLTHQGRSSSSGHYVSWVKRKQDEWIKFDDDKVSIVTPEDILRLSGGGDWHIAYVLLYGPRRVEIMEEESEQ",
# EPHB1_HUMAN
"MALDYLLLLLLASAVAAMEETLMDTRTATAELGWTANPASGWEEVSGYDENLNTIRTYQVCNVFEPNQNNWLLTTFINRRGAHRIYTEMRFTVRDCSSLPNVPGSCKETFNLYYYETDSVIATKKSAFWSEAPYLKVDTIAADESFSQVDFGGRLMKVNTEVRSFGPLTRNGFYLAFQDYGACMSLLSVRVFFKKCPSIVQNFAVFPETMTGAESTSLVIARGTCIPNAEEVDVPIKLYCNGDGEWMVPIGRCTCKPGYEPENSVACKACPAGTFKASQEAEGCSHCPSNSRSPAEASPICTCRTGYYRADFDPPEVACTSVPSGPRNVISIVNETSIILEWHPPRETGGRDDVTYNIICKKCRADRRSCSRCDDNVEFVPRQLGLTECRVSISSLWAHTPYTFDIQAINGVSSKSPFPPQHVSVNITTNQAAPSTVPIMHQVSATMRSITLSWPQPEQPNGIILDYEIRYYEKEHNEFNSSMARSQTNTARIDGLRPGMVYVVQVRARTVAGYGKFSGKMCFQTLTDDDYKSELREQLPLIAGSAAAGVVFVVSLVAISIVCSRKRAYSKEAVYSDKLQHYSTGRGSPGMKIYIDPFTYEDPNEAVREFAKEIDVSFVKIEEVIGAGEFGEVYKGRLKLPGKREIYVAIKTLKAGYSEKQRRDFLSEASIMGQFDHPNIIRLEGVVTKSRPVMIITEFMENGALDSFLRQNDGQFTVIQLVGMLRGIAAGMKYLAEMNYVHRDLAARNILVNSNLVCKVSDFGLSRYLQDDTSDPTYTSSLGGKIPVRWTAPEAIAYRKFTSASDVWSYGIVMWEVMSFGERPYWDMSNQDVINAIEQDYRLPPPMDCPAALHQLMLDCWQKDRNSRPRFAEIVNTLDKMIRNPASLKTVATITAVPSQPLLDRSIPDFTAFTTVDDWLSAIKMVQYRDSFLTAGFTSLQLVTQMTSEDLLRIGITLAGHQKKILNSIHSMRVQISQSPTAMA",
# EPHA4_HUMAN
"MAGIFYFALFSCLFGICDAVTGSRVYPANEVTLLDSRSVQGELGWIASPLEGGWEEVSIMDEKNTPIRTYQVCNVMEPSQNNWLRTDWITREGAQRVYIEIKFTLRDCNSLPGVMGTCKETFNLYYYESDNDKERFIRENQFVKIDTIAADESFTQVDIGDRIMKLNTEIRDVGPLSKKGFYLAFQDVGACIALVSVRVFYKKCPLTVRNLAQFPDTITGADTSSLVEVRGSCVNNSEEKDVPKMYCGADGEWLVPIGNCLCNAGHEERSGECQACKIGYYKALSTDATCAKCPPHSYSVWEGATSCTCDRGFFRADNDAASMPCTRPPSAPLNLISNVNETSVNLEWSSPQNTGGRQDISYNVVCKKCGAGDPSKCRPCGSGVHYTPQQNGLKTTKVSITDLLAHTNYTFEIWAVNGVSKYNPNPDQSVSVTVTTNQAAPSSIALVQAKEVTRYSVALAWLEPDRPNGVILEYEVKYYEKDQNERSYRIVRTAARNTDIKGLNPLTSYVFHVRARTAAGYGDFSEPLEVTTNTVPSRIIGDGANSTVLLVSVSGSVVLVVILIAAFVISRRRSKYSKAKQEADEEKHLNQGVRTYVDPFTYEDPNQAVREFAKEIDASCIKIEKVIGVGEFGEVCSGRLKVPGKREICVAIKTLKAGYTDKQRRDFLSEASIMGQFDHPNIIHLEGVVTKCKPVMIITEYMENGSLDAFLRKNDGRFTVIQLVGMLRGIGSGMKYLSDMSYVHRDLAARNILVNSNLVCKVSDFGMSRVLEDDPEAAYTTRGGKIPIRWTAPEAIAYRKFTSASDVWSYGIVMWEVMSYGERPYWDMSNQDVIKAIEEGYRLPPPMDCPIALHQLMLDCWQKERSDRPKFGQIVNMLDKLIRNPNSLKRTGTESSRPNTALLDPSSPEFSAVVSVGDWLQAIKMDRYKDNFTAAGYTTLEAVVHVNQEDLARIGITAITHQNKILSSVQAMRTQMQQMHGRMVPV",
# DVLP1_HUMAN
"MAETKIIYHMDEEETPYLVKLPVAPERVTLADFKNVLSNRPVHAYKFFFKSMDQDFGVVKEEIFDDNAKLPCFNGRVVSWLVLVEGAHSDAGSQGTDSHTDLPPPLERTGGIGDSRSPSFQPDVASSRDGMDNETGTESMVSHRRDRARRRNREEAARTNGHPRGDRRRDVGLPPDSASTALSSELESSSFVDSDEDDSTSRLSSSTEQSTSSRLIRKHKRRRRKQRLRQADRASSFSSMTDSTMSLNIITVTLNMERHHFLGICIVGQSNDRGDGGIYIGSIMKGGAVAADGRIEPGDMLLQVNDVNFENMSNDDAVRVLREIVSQTGPISLTVAKCWDPTPRSYFTVPRPDPVRPIDPAAWLSHTAALTGALPRPQLEEAPLTVESDMNTVVRVMQLPDSGLEIRDRMWLKITIANAVIGADVVDWLYTHVEGFKERREARKYASSLLKHGFLRHTVNKITFSEQCYYVFGDLCSNLATLNLNSGSSGTSDQDTLAPLPHPAAPWPLGQGYPYQYPGPPPCFPPAYQDPGFSYGSGSTGSQQSEGSKSSGSTRNTLRPPACEKERRAAGSGDSDSESDHTAPSGVGSSWRERPADQLSRGSSPRSQASSYAPGLPPPHPTTKAYTVVGGPPGGPPVRELAAVPPELTGSRQSFQKAMGNPCEFFVDIM",
# SNAA_HUMAN
"MDNSGKEAEAMALLAEAERKVKNSQSFFSGLFGGSSKIEEACEIYARAANMFKMAKNWSAAGNAFCQAAQLHLQLQSKHDAATCFVDAGNAFKKADPQEAINCLMRAIEIYTDMGRFTIAAKHHISIAEIYETELVDIEKAIAHYEQSADYYKGEESNSSANKCLLKVAGYAALLEQYQKAIDIYEQVGTNAMDSPLLKYSAKDYFFKAALCHFCIDMLNAKLAVQKYEELFPAFSDSRECKLMKKLLEAHEEQNVDSYTESVKEYDSISRLDQWLTTMLLRIKKTIQGDEEDLR",
# IF5_HUMAN
"MSVNVNRSVSDQFYRYKMPRLIAKVEGKGNGIKTVIVNMVDVAKALNRPPTYPTKYFGCELGAQTQFDVKNDRYIVNGSHEANKLQDMLDGFIKKFVLCPECENPETDLHVNPKKQTIGNSCKACGYRGMLDTHHKLCTFILKNPPENSDSGTGKKEKEKKNRKGKDKENGSVSSSETPPPPPPPNEINPPPHTMEEEEDDDWGEDTTEEAQRRRMDEISDHAKVLTLSDDLERTIEERVNILFDFVKKKKEEGVIDSSDKEIVAEAERLDVKAMGPLVLTEVLFNEKIREQIKKYRRHFLRFCHNNKKAQRYLLHGLECVVAMHQAQLISKIPHILKEMYDADLLEEEVIISWSEKASKKYVSKELAKEIRVKAEPFIKWLKEAEEESSGGEEEDEDENIEVVYSKAASVPKVETVKSDNKDDDIDIDAI",
# TERA_HUMAN
"MASGADSKGDDLSTAILKQKNRPNRLIVDEAINEDNSVVSLSQPKMDELQLFRGDTVLLKGKKRREAVCIVLSDDTCSDEKIRMNRVVRNNLRVRLGDVISIQPCPDVKYGKRIHVLPIDDTVEGITGNLFEVYLKPYFLEAYRPIRKGDIFLVRGGMRAVEFKVVETDPSPYCIVAPDTVIHCEGEPIKREDEEESLNEVGYDDIGGCRKQLAQIKEMVELPLRHPALFKAIGVKPPRGILLYGPPGTGKTLIARAVANETGAFFFLINGPEIMSKLAGESESNLRKAFEEAEKNAPAIIFIDELDAIAPKREKTHGEVERRIVSQLLTLMDGLKQRAHVIVMAATNRPNSIDPALRRFGRFDREVDIGIPDATGRLEILQIHTKNMKLADDVDLEQVANETHGHVGADLAALCSEAALQAIRKKMDLIDLEDETIDAEVMNSLAVTMDDFRWALSQSNPSALRETVVEVPQVTWEDIGGLEDVKRELQELVQYPVEHPDKFLKFGMTPSKGVLFYGPPGCGKTLLAKAIANECQANFISIKGPELLTMWFGESEANVREIFDKARQAAPCVLFFDELDSIAKARGGNIGDGGGAADRVINQILTEMDGMSTKKNVFIIGATNRPDIIDPAILRPGRLDQLIYIPLPDEKSRVAILKANLRKSPVAKDVDLEFLAKMTNGFSGADLTEICQRACKLAIRESIESEIRRERERQTNPSAMEVEEDDPVPEIRRDHFEEAMRFARRSVSDNDIRKYEMFAQTLQQSRGFGSFRFPSGNQGGAGPSQGSGGGTGGSVYTEDNDDDLYG",
# AFAD_HUMAN
"MSAGGRDEERRKLADIIHHWNANRLDLFEISQPTEDLEFHGVMRFYFQDKAAGNFATKCIRVSSTATTQDVIETLAEKFRPDMRMLSSPKYSLYEVHVSGERRLDIDEKPLVVQLNWNKDDREGRFVLKNENDAIPPKKAQSNGPEKQEKEGVIQNFKRTLSKKEKKEKKKREKEALRQASDKDDRPFQGEDVENSRLAAEVYKDMPETSFTRTISNPEVVMKRRRQQKLEKRMQEFRSSDGRPDSGGTLRIYADSLKPNIPYKTILLSTTDPADFAVAEALEKYGLEKENPKDYCIARVMLPPGAQHSDEKGAKEIILDDDECPLQIFREWPSDKGILVFQLKRRPPDHIPKKTKKHLEGKTPKGKERADGSGYGSTLPPEKLPYLVELSPGRRNHFAYYNYHTYEDGSDSRDKPKLYRLQLSVTEVGTEKLDDNSIQLFGPGIQPHHCDLTNMDGVVTVTPRSMDAETYVEGQRISETTMLQSGMKVQFGASHVFKFVDPSQDHALAKRSVDGGLMVKGPRHKPGIVQETTFDLGGDIHSGTALPTSKSTTRLDSDRVSSASSTAERGMVKPMIRVEQQPDYRRQESRTQDASGPELILPASIEFRESSEDSFLSAIINYTNSSTVHFKLSPTYVLYMACRYVLSNQYRPDISPTERTHKVIAVVNKMVSMMEGVIQKQKNIAGALAFWMANASELLNFIKQDRDLSRITLDAQDVLAHLVQMAFKYLVHCLQSELNNYMPAFLDDPEENSLQRPKIDDVLHTLTGAMSLLRRCRVNAALTIQLFSQLFHFINMWLFNRLVTDPDSGLCSHYWGAIIRQQLGHIEAWAEKQGLELAADCHLSRIVQATTLLTMDKYAPDDIPNINSTCFKLNSLQLQALLQNYHCAPDEPFIPTDLIENVVTVAENTADELARSDGREVQLEEDPDLQLPFLLPEDGYSCDVVRNIPNGLQEFLDPLCQRGFCRLIPHTRSPGTWTIYFEGADYESHLLRENTELAQPLRKEPEIITVTLKKQNGMGLSIVAAKGAGQDKLGIYVKSVVKGGAADVDGRLAAGDQLLSVDGRSLVGLSQERAAELMTRTSSVVTLEVAKQGAIYHGLATLLNQPSPMMQRISDRRGSGKPRPKSEGFELYNNSTQNGSPESPQLPWAEYSEPKKLPGDDRLMKNRADHRSSPNVANQPPSPGGKSAYASGTTAKITSVSTGNLCTEEQTPPPRPEAYPIPTQTYTREYFTFPASKSQDRMAPPQNQWPNYEEKPHMHTDSNHSSIAIQRVTRSQEELREDKAYQLERHRIEAAMDRKSDSDMWINQSSSLDSSTSSQEHLNHSSKSVTPASTLTKSGPGRWKTPAAIPATPVAVSQPIRTDLPPPPPPPPVHYAGDFDGMSMDLPLPPPPSANQIGLPSAQVAAAERRKREEHQRWYEKEKARLEEERERKRREQERKLGQMRTQSLNPAPFSPLTAQQMKPEKPSTLQRPQETVIRELQPQQQPRTIERRDLQYITVSKEELSSGDSLSPDPWKRDAKEKLEKQQQMHIVDMLSKEIQELQSKPDRSAEESDRLRKLMLEWQFQKRLQESKQKDEDDEEEEDDDVDTMLIMQRLEAERRARLQDEERRRQQQLEEMRKREAEDRARQEEERRRQEEERTKRDAEEKRRQEEGYYSRLEAERRRQHDEAARRLLEPEAPGLCRPPLPRDYEPPSPSPAPGAPPPPPQRNASYLKTQVLSPDSLFTAKFVAYNEEEEEEDCSLAGPNSYPGSTGAAVGAHDACRDAKEKRSKSQDADSPGSSGAPENLTFKERQRLFSQGQDVSNKVKASRKLTELENELNTK",
# LAMB2_HUMAN
"MELTSRERGRGQPLPWELRLGLLLSVLAATLAQAPAPDVPGCSRGSCYPATGDLLVGRADRLTASSTCGLNGPQPYCIVSHLQDEKKCFLCDSRRPFSARDNPHSHRIQNVVTSFAPQRRAAWWQSENGIPAVTIQLDLEAEFHFTHLIMTFKTFRPAAMLVERSADFGRTWHVYRYFSYDCGADFPGVPLAPPRHWDDVVCESRYSEIEPSTEGEVIYRVLDPAIPIPDPYSSRIQNLLKITNLRVNLTRLHTLGDNLLDPRREIREKYYYALYELVVRGNCFCYGHASECAPAPGAPAHAEGMVHGACICKHNTRGLNCEQCQDFYRDLPWRPAEDGHSHACRKCECHGHTHSCHFDMAVYLASGNVSGGVCDGCQHNTAGRHCELCRPFFYRDPTKDLRDPAVCRSCDCDPMGSQDGGRCDSHDDPALGLVSGQCRCKEHVVGTRCQQCRDGFFGLSISDRLGCRRCQCNARGTVPGSTPCDPNSGSCYCKRLVTGRGCDRCLPGHWGLSHDLLGCRPCDCDVGGALDPQCDEGTGQCHCRQHMVGRRCEQVQPGYFRPFLDHLIWEAEDTRGQVLDVVERLVTPGETPSWTGSGFVRLQEGQTLEFLVASVPKAMDYDLLLRLEPQVPEQWAELELIVQRPGPVPAHSLCGHLVPKDDRIQGTLQPHARYLIFPNPVCLEPGISYKLHLKLVRTGGSAQPETPYSGPGLLIDSLVLLPRVLVLEMFSGGDAAALERQATFERYQCHEEGLVPSKTSPSEACAPLLISLSTLIYNGALPCQCNPQGSLSSECNPHGGQCLCKPGVVGRRCDLCAPGYYGFGPTGCQACQCSHEGALSSLCEKTSGQCLCRTGAFGLRCDRCQRGQWGFPSCRPCVCNGHADECNTHTGACLGCRDHTGGEHCERCIAGFHGDPRLPYGGQCRPCPCPEGPGSQRHFATSCHQDEYSQQIVCHCRAGYTGLRCEACAPGHFGDPSRPGGRCQLCECSGNIDPMDPDACDPHTGQCLRCLHHTEGPHCAHCKPGFHGQAARQSCHRCTCNLLGTNPQQCPSPDQCHCDPSSGQCPCLPNVQGPSCDRCAPNFWNLTSGHGCQPCACHPSRARGPTCNEFTGQCHCRAGFGGRTCSECQELHWGDPGLQCHACDCDSRGIDTPQCHRFTGHCSCRPGVSGVRCDQCARGFSGIFPACHPCHACFGDWDRVVQDLAARTQRLEQRAQELQQTGVLGAFESSFWHMQEKLGIVQGIVGARNTSAASTAQLVEATEELRREIGEATEHLTQLEADLTDVQDENFNANHALSGLERDRLALNLTLRQLDQHLDLLKHSNFLGAYDSIRHAHSQSAEAERRANTSALAVPSPVSNSASARHRTEALMDAQKEDFNSKHMANQRALGKLSAHTHTLSLTDINELVCGAPGDAPCATSPCGGAGCRDEDGQPRCGGLSCNGAAATADLALGRARHTQAELQRALAEGGSILSRVAETRRQASEAQQRAQAALDKANASRGQVEQANQELQELIQSVKDFLNQEGADPDSIEMVATRVLELSIPASAEQIQHLAGAIAERVRSLADVDAILARTVGDVRRAEQLLQDARRARSWAEDEKQKAETVQAALEEAQRAQGIAQGAIRGAVADTRDTEQTLYQVQERMAGAERALSSAGERARQLDALLEALKLKRAGNSLAASTAEETAGSAQGRAQEAEQLLRGPLGDQYQTVKALAERKAQGVLAAQARAEQLRDEARDLLQAAQDKLQRLQELEGTYEENERALESKAAQLDGLEARMRSVLQAINLQVQIYNTCQ",
# CADH6_HUMAN
"MRTYRYFLLLFWVGQPYPTLSTPLSKRTSGFPAKKRALELSGNSKNELNRSKRSWMWNQFFLLEEYTGSDYQYVGKLHSDQDRGDGSLKYILSGDGAGDLFIINENTGDIQATKRLDREEKPVYILRAQAINRRTGRPVEPESEFIIKIHDINDNEPIFTKEVYTATVPEMSDVGTFVVQVTATDADDPTYGNSAKVVYSILQGQPYFSVESETGIIKTALLNMDRENREQYQVVIQAKDMGGQMGGLSGTTTVNITLTDVNDNPPRFPQSTYQFKTPESSPPGTPIGRIKASDADVGENAEIEYSITDGEGLDMFDVITDQETQEGIITVKKLLDFEKKKVYTLKVEASNPYVEPRFLYLGPFKDSATVRIVVEDVDEPPVFSKLAYILQIREDAQINTTIGSVTAQDPDAARNPVKYSVDRHTDMDRIFNIDSGNGSIFTSKLLDRETLLWHNITVIATEINNPKQSSRVPLYIKVLDVNDNAPEFAEFYETFVCEKAKADQLIQTLHAVDKDDPYSGHQFSFSLAPEAASGSNFTIQDNKDNTAGILTRKNGYNRHEMSTYLLPVVISDNDYPVQSSTGTVTVRVCACDHHGNMQSCHAEALIHPTGLSTGALVAILLCIVILLVTVVLFAALRRQRKKEPLIISKEDIRDNIVSYNDEGGGEEDTQAFDIGTLRNPEAIEDNKLRRDIVPEALFLPRRTPTARDNTDVRDFINQRLKENDTDPTAPPYDSLATYAYEGTGSVADSLSSLESVTTDADQDYDYLSDWGPRFKKLADMYGGVDSDKDS",
# CADH8_HUMAN
"MPERLAEMLLDLWTPLIILWITLPPCIYMAPMNQSQVLMSGSPLELNSLGEEQRILNRSKRGWVWNQMFVLEEFSGPEPILVGRLHTDLDPGSKKIKYILSGDGAGTIFQINDVTGDIHAIKRLDREEKAEYTLTAQAVDWETSKPLEPPSEFIIKVQDINDNAPEFLNGPYHATVPEMSILGTSVTNVTATDADDPVYGNSAKLVYSILEGQPYFSIEPETAIIKTALPNMDREAKEEYLVVIQAKDMGGHSGGLSGTTTLTVTLTDVNDNPPKFAQSLYHFSVPEDVVLGTAIGRVKANDQDIGENAQSSYDIIDGDGTALFEITSDAQAQDGIIRLRKPLDFETKKSYTLKVEAANVHIDPRFSGRGPFKDTATVKIVVEDADEPPVFSSPTYLLEVHENAALNSVIGQVTARDPDITSSPIRFSIDRHTDLERQFNINADDGKITLATPLDRELSVWHNITIIATEIRNHSQISRVPVAIKVLDVNDNAPEFASEYEAFLCENGKPGQVIQTVSAMDKDDPKNGHYFLYSLLPEMVNNPNFTIKKNEDNSLSILAKHNGFNRQKQEVYLLPIIISDSGNPPLSSTSTLTIRVCGCSNDGVVQSCNVEAYVLPIGLSMGALIAILACIILLLVIVVLFVTLRRHKNEPLIIKDDEDVRENIIRYDDEGGGEEDTEAFDIATLQNPDGINGFLPRKDIKPDLQFMPRQGLAPVPNGVDVDEFINVRLHEADNDPTAPPYDSIQIYGYEGRGSVAGSLSSLESTTSDSDQNFDYLSDWGPRFKRLGELYSVGESDKET",
# CAD11_HUMAN
"MKENYCLQAALVCLGMLCHSHAFAPERRGHLRPSFHGHHEKGKEGQVLQRSKRGWVWNQFFVIEEYTGPDPVLVGRLHSDIDSGDGNIKYILSGEGAGTIFVIDDKSGNIHATKTLDREERAQYTLMAQAVDRDTNRPLEPPSEFIVKVQDINDNPPEFLHETYHANVPERSNVGTSVIQVTASDADDPTYGNSAKLVYSILEGQPYFSVEAQTGIIRTALPNMDREAKEEYHVVIQAKDMGGHMGGLSGTTKVTITLTDVNDNPPKFPQSVYQMSVSEAAVPGEEVGRVKAKDPDIGENGLVTYNIVDGDGMESFEITTDYETQEGVIKLKKPVDFETKRAYSLKVEAANVHIDPKFISNGPFKDTVTVKISVEDADEPPMFLAPSYIHEVQENAAAGTVVGRVHAKDPDAANSPIRYSIDRHTDLDRFFTINPEDGFIKTTKPLDREETAWLNITVFAAEIHNRHQEAKVPVAIRVLDVNDNAPKFAAPYEGFICESDQTKPLSNQPIVTISADDKDDTANGPRFIFSLPPEIIHNPNFTVRDNRDNTAGVYARRGGFSRQKQDLYLLPIVISDGGIPPMSSTNTLTIKVCGCDVNGALLSCNAEAYILNAGLSTGALIAILACIVILLVIVVLFVTLRRQKKEPLIVFEEEDVRENIITYDDEGGGEEDTEAFDIATLQNPDGINGFIPRKDIKPEYQYMPRPGLRPAPNSVDVDDFINTRIQEADNDPTAPPYDSIQIYGYEGRGSVAGSLSSLESATTDSDLDYDYLQNWGPRFKKLADLYGSKDTFDDDS",
# CAD13_HUMAN
"MQPRTPLVLCVLLSQVLLLTSAEDLDCTPGFQQKVFHINQPAEFIEDQSILNLTFSDCKGNDKLRYEVSSPYFKVNSDGGLVALRNITAVGKTLFVHARTPHAEDMAELVIVGGKDIQGSLQDIFKFARTSPVPRQKRSIVVSPILIPENQRQPFPRDVGKVVDSDRPERSKFRLTGKGVDQEPKGIFRINENTGSVSVTRTLDREVIAVYQLFVETTDVNGKTLEGPVPLEVIVIDQNDNRPIFREGPYIGHVMEGSPTGTTVMRMTAFDADDPATDNALLRYNIRQQTPDKPSPNMFYIDPEKGDIVTVVSPALLDRETLENPKYELIIEAQDMAGLDVGLTGTATATIMIDDKNDHSPKFTKKEFQATVEEGAVGVIVNLTVEDKDDPTTGAWRAAYTIINGNPGQSFEIHTNPQTNEGMLSVVKPLDYEISAFHTLLIKVENEDPLVPDVSYGPSSTATVHITVLDVNEGPVFYPDPMMVTRQEDLSVGSVLLTVNATDPDSLQHQTIRYSVYKDPAGWLNINPINGTVDTTAVLDRESPFVDNSVYTALFLAIDSGNPPATGTGTLLITLEDVNDNAPFIYPTVAEVCDDAKNLSVVILGASDKDLHPNTDPFKFEIHKQAVPDKVWKISKINNTHALVSLLQNLNKANYNLPIMVTDSGKPPMTNITDLRVQVCSCRNSKVDCNAAGALRFSLPSVLLLSLFSLACL",
# TPD52_HUMAN
"MDCREMDLYEDYQSPFDFDAGVNKSYLYLSPSGNSSPPGSPTLQKFGLLRTDPVPEEGEDVAATISATETLSEEEQEELRRELAKVEEEIQTLSQVLAAKEKHLAEIKRKLGINSLQELKQNIAKGWQDVTATSAYKKTSETLSQAGQKASAAFSSVGSVITKKLEDVKNSPTFKSFEEKVENLKSKVGGTKPAGGDFGEVLNSAANASATTTEPLPEKTQESL",
# HNRH2_HUMAN
"MMLSTEGREGFVVKVRGLPWSCSADEVMRFFSDCKIQNGTSGIRFIYTREGRPSGEAFVELESEEEVKLALKKDRETMGHRYVEVFKSNSVEMDWVLKHTGPNSPDTANDGFVRLRGLPFGCSKEEIVQFFSGLEIVPNGMTLPVDFQGRSTGEAFVQFASQEIAEKALKKHKERIGHRYIEIFKSSRAEVRTHYDPPRKLMAMQRPGPYDRPGAGRGYNSIGRGAGFERMRRGAYGGGYGGYDDYGGYNDGYGFGSDRFGRDLNYCFSGMSDHRYGDGGSSFQSTTGHCVHMRGLPYRATENDIYNFFSPLNPMRVHIEIGPDGRVTGEADVEFATHEDAVAAMAKDKANMQHRYVELFLNSTAGTSGGAYDHSYVELFLNSTAGASGGAYGSQMMGGMGLSNQSSYGGPASQQLSGGYGGGYGGQSSMSGYDQVLQENSSDYQSNLA",
# SUMO3_HUMAN
"MSEEKPKEGVKTENDHINLKVAGQDGSVVQFKIKRHTPLSKLMKAYCERQGLSMRQIRFRFDGQPINETDTPAQLEMEDEDTIDVFQQQTGGVPESSLAGHSF",
# EIF3B_HUMAN
"MQDAENVAVPEAAEERAEPGQQQPAAEPPPAEGLLRPAGPGAPEAAGTEASSEEVGIAEAGPESEVRTEPAAEAEAASGPSESPSPPAAEELPGSHAEPPVPAQGEAPGEQARDERSDSRAQAVSEDAGGNEGRAAEAEPRALENGDADEPSFSDPEDFVDDVSEEELLGDVLKDRPQEADGIDSVIVVDNVPQVGPDRLEKLKNVIHKIFSKFGKITNDFYPEEDGKTKGYIFLEYASPAHAVDAVKNADGYKLDKQHTFRVNLFTDFDKYMTISDEWDIPEKQPFKDLGNLRYWLEEAECRDQYSVIFESGDRTSIFWNDVKDPVSIEERARWTETYVRWSPKGTYLATFHQRGIALWGGEKFKQIQRFSHQGVQLIDFSPCERYLVTFSPLMDTQDDPQAIIIWDILTGHKKRGFHCESSAHWPIFKWSHDGKFFARMTLDTLSIYETPSMGLLDKKSLKISGIKDFSWSPGGNIIAFWVPEDKDIPARVTLMQLPTRQEIRVRNLFNVVDCKLHWQKNGDYLCVKVDRTPKGTQGVVTNFEIFRMREKQVPVDVVEMKETIIAFAWEPNGSKFAVLHGEAPRISVSFYHVKNNGKIELIKMFDKQQANTIFWSPQGQFVVLAGLRSMNGALAFVDTSDCTVMNIAEHYMASDVEWDPTGRYVVTSVSWWSHKVDNAYWLWTFQGRLLQKNNKDRFCQLLWRPRPPTLLSQEQIKQIKKDLKKYSKIFEQKDRLSQSKASKELVERRRTMMEDFRKYRKMAQELYMEQKNERLELRGGVDTDELDSNVDDWEEETIEFFVTEEIIPLGNQE",
# P2RX3_HUMAN
"MNCISDFFTYETTKSVVVKSWTIGIINRVVQLLIISYFVGWVFLHEKAYQVRDTAIESSVVTKVKGSGLYANRVMDVSDYVTPPQGTSVFVIITKMIVTENQMQGFCPESEEKYRCVSDSQCGPERLPGGGILTGRCVNYSSVLRTCEIQGWCPTEVDTVETPIMMEAENFTIFIKNSIRFPLFNFEKGNLLPNLTARDMKTCRFHPDKDPFCPILRVGDVVKFAGQDFAKLARTGGVLGIKIGWVCDLDKAWDQCIPKYSFTRLDSVSEKSSVSPGYNFRFAKYYKMENGSEYRTLLKAFGIRFDVLVYGNAGKFNIIPTIISSVAAFTSVGVGTVLCDIILLNFLKGADQYKAKKFEEVNETTLKIAALTNPVYPSDQTTAEKQSTDSGAFSIGH",
# AP1S2_HUMAN
"MQFMLLFSRQGKLRLQKWYVPLSDKEKKKITRELVQTVLARKPKMCSFLEWRDLKIVYKRYASLYFCCAIEDQDNELITLEIIHRYVELLDKYFGSVCELDIIFNFEKAYFILDEFLLGGEVQETSKKNVLKAIEQADLLQEEAETPRSVLEEIGLT",
# IF6_HUMAN
"MAVRASFENNCEIGCFAKLTNTYCLVAIGGSENFYSVFEGELSDTIPVVHASIAGCRIIGRMCVGNRHGLLVPNNTTDQELQHIRNSLPDTVQIRRVEERLSALGNVTTCNDYVALVHPDLDRETEEILADVLKVEVFRQTVADQVLVGSYCVFSNQGGLVHPKTSIEDQDELSSLLQVPLVAGTVNRGSEVIAAGMVVNDWCAFCGLDTTSTELSVVESVFKLNEAQPSTIATSMRDSLIDSLT",
# CTBP2_HUMAN
"MALVDKHKVKRQRLDRICEGIRPQIMNGPLHPRPLVALLDGRDCTVEMPILKDLATVAFCDAQSTQEIHEKVLNEAVGAMMYHTITLTREDLEKFKALRVIVRIGSGYDNVDIKAAGELGIAVCNIPSAAVEETADSTICHILNLYRRNTWLYQALREGTRVQSVEQIREVASGAARIRGETLGLIGFGRTGQAVAVRAKAFGFSVIFYDPYLQDGIERSLGVQRVYTLQDLLYQSDCVSLHCNLNEHNHHLINDFTIKQMRQGAFLVNAARGGLVDEKALAQALKEGRIRGAALDVHESEPFSFAQGPLKDAPNLICTPHTAWYSEQASLEMREAAATEIRRAITGRIPESLRNCVNKEFFVTSAPWSVIDQQAIHPELNGATYRYPPGIVGVAPGGLPAAMEGIIPGGIPVTHNLPTVAHPSQAPSPNQPTKHGDNREHPNEQ",
# WNT3A_HUMAN
"MAPLGYFLLLCSLKQALGSYPIWWSLAVGPQYSSLGSQPILCASIPGLVPKQLRFCRNYVEIMPSVAEGIKIGIQECQHQFRGRRWNCTTVHDSLAIFGPVLDKATRESAFVHAIASAGVAFAVTRSCAEGTAAICGCSSRHQGSPGKGWKWGGCSEDIEFGGMVSREFADARENRPDARSAMNRHNNEAGRQAIASHMHLKCKCHGLSGSCEVKTCWWSQPDFRAIGDFLKDKYDSASEMVVEKHRESRGWVETLRPRYTYFKVPTERDLVYYEASPNFCEPNPETGSFGTRDRTCNVSSHGIDGCDLLCCGRGHNARAERRREKCRCVFHWCCYVSCQECTRVYDVHTCK",
# NETR_HUMAN
"MTLARFVLALMLGALPEVVGFDSVLNDSLHHSHRHSPPAGPHYPYYLPTQQRPPRTRPPPPLPRFPRPPRALPAQRPHALQAGHTPRPHPWGCPAGEPWVSVTDFGAPCLRWAEVPPFLERSPPASWAQLRGQRHNFCRSPDGAGRPWCFYGDARGKVDWGYCDCRHGSVRLRGGKNEFEGTVEVYASGVWGTVCSSHWDDSDASVICHQLQLGGKGIAKQTPFSGLGLIPIYWSNVRCRGDEENILLCEKDIWQGGVCPQKMAAAVTCSFSHGPTFPIIRLAGGSSVHEGRVELYHAGQWGTVCDDQWDDADAEVICRQLGLSGIAKAWHQAYFGEGSGPVMLDEVRCTGNELSIEQCPKSSWGEHNCGHKEDAGVSCTPLTDGVIRLAGGKGSHEGRLEVYYRGQWGTVCDDGWTELNTYVVCRQLGFKYGKQASANHFEESTGPIWLDDVSCSGKETRFLQCSRRQWGRHDCSHREDVSIACYPGGEGHRLSLGFPVRLMDGENKKEGRVEVFINGQWGTICDDGWTDKDAAVICRQLGYKGPARARTMAYFGEGKGPIHVDNVKCTGNERSLADCIKQDIGRHNCRHSEDAGVICDYFGKKASGNSNKESLSSVCGLRLLHRRQKRIIGGKNSLRGGWPWQVSLRLKSSHGDGRLLCGATLLSSCWVLTAAHCFKRYGNSTRSYAVRVGDYHTLVPEEFEEEIGVQQIVIHREYRPDRSDYDIALVRLQGPEEQCARFSSHVLPACLPLWRERPQKTASNCYITGWGDTGRAYSRTLQQAAIPLLPKRFCEERYKGRFTGRMLCAGNLHEHKRVDSCQGDSGGPLMCERPGESWVVYGVTSWGYGCGVKDSPGVYTKVSAFVPWIKSVTKL",
# BACE1_HUMAN
"MAQALPWLLLWMGAGVLPAHGTQHGIRLPLRSGLGGAPLGLRLPRETDEEPEEPGRRGSFVEMVDNLRGKSGQGYYVEMTVGSPPQTLNILVDTGSSNFAVGAAPHPFLHRYYQRQLSSTYRDLRKGVYVPYTQGKWEGELGTDLVSIPHGPNVTVRANIAAITESDKFFINGSNWEGILGLAYAEIARPDDSLEPFFDSLVKQTHVPNLFSLQLCGAGFPLNQSEVLASVGGSMIIGGIDHSLYTGSLWYTPIRREWYYEVIIVRVEINGQDLKMDCKEYNYDKSIVDSGTTNLRLPKKVFEAAVKSIKAASSTEKFPDGFWLGEQLVCWQAGTTPWNIFPVISLYLMGEVTNQSFRITILPQQYLRPVEDVATSQDDCYKFAISQSSTGTVMGAVIMEGFYVVFDRARKRIGFAVSACHVHDEFRTAAVEGPFVTLDMEDCGYNIPQTDESTLMTIAYVMAAICALFMLPLCLMVCQWRCLRCLRQQHDDFADDISLLK",
# NRG3_HUMAN
"MSEGAAAASPPGAASAAAASAEEGTAAAAAAAAAGGGPDGGGEGAAEPPRELRCSDCIVWNRQQTWLCVVPLFIGFIGLGLSLMLLKWIVVGSVKEYVPTDLVDSKGMGQDPFFLSKPSSFPKAMETTTTTTSTTSPATPSAGGAASSRTPNRISTRLTTITRAPTRFPGHRVPIRASPRSTTARNTAAPATVPSTTAPFFSSSTLGSRPPVPGTPSTQAMPSWPTAAYATSSYLHDSTPSWTLSPFQDAASSSSSSSSSATTTTPETSTSPKFHTTTYSTERSEHFKPCRDKDLAYCLNDGECFVIETLTGSHKHCRCKEGYQGVRCDQFLPKTDSILSDPTDHLGIEFMESEEVYQRQVLSISCIIFGIVIVGMFCAAFYFKSKKQAKQIQEQLKVPQNGKSYSLKASSTMAKSENLVKSHVQLQNYSKVERHPVTALEKMMESSFVGPQSFPEVPSPDRGSQSVKHHRSLSSCCSPGQRSGMLHRNAFRRTPPSPRSRLGGIVGPAYQQLEESRIPDQDTIPCQGIEVRKTISHLPIQLWCVERPLDLKYSSSGLKTQRNTSINMQLPSRETNPYFNSLEQKDLVGYSSTRASSVPIIPSVGLEETCLQMPGISEVKSIKWCKNSYSADVVNVSIPVSDCLIAEQQEVKILLETVQEQIRILTDARRSEDYELASVETEDSASENTAFLPLSPTAKSEREAQFVLRNEIQRDSALTK",
# NAC3_HUMAN
"MAWLRLQPLTSAFLHFGLVTFVLFLNGLRAEAGGSGDVPSTGQNNESCSGSSDCKEGVILPIWYPENPSLGDKIARVIVYFVALIYMFLGVSIIADRFMASIEVITSQEREVTIKKPNGETSTTTIRVWNETVSNLTLMALGSSAPEILLSLIEVCGHGFIAGDLGPSTIVGSAAFNMFIIIGICVYVIPDGETRKIKHLRVFFITAAWSIFAYIWLYMILAVFSPGVVQVWEGLLTLFFFPVCVLLAWVADKRLLFYKYMHKKYRTDKHRGIIIETEGDHPKGIEMDGKMMNSHFLDGNLVPLEGKEVDESRREMIRILKDLKQKHPEKDLDQLVEMANYYALSHQQKSRAFYRIQATRMMTGAGNILKKHAAEQAKKASSMSEVHTDEPEDFISKVFFDPCSYQCLENCGAVLLTVVRKGGDMSKTMYVDYKTEDGSANAGADYEFTEGTVVLKPGETQKEFSVGIIDDDIFEEDEHFFVRLSNVRIEEEQPEEGMPPAIFNSLPLPRAVLASPCVATVTILDDDHAGIFTFECDTIHVSESIGVMEVKVLRTSGARGTVIVPFRTVEGTAKGGGEDFEDTYGELEFKNDETVKTIRVKIVDEEEYERQENFFIALGEPKWMERGISALLLSPDVTDRKLTMEEEEAKRIAEMGKPVLGEHPKLEVIIEESYEFKTTVDKLIKKTNLALVVGTHSWRDQFMEAITVSAAGDEDEDESGEERLPSCFDYVMHFLTVFWKVLFACVPPTEYCHGWACFAVSILIIGMLTAIIGDLASHFGCTIGLKDSVTAVVFVAFGTSVPDTFASKAAALQDVYADASIGNVTGSNAVNVFLGIGLAWSVAAIYWALQGQEFHVSAGTLAFSVTLFTIFAFVCISVLLYRRRPHLGGELGGPRGCKLATTWLFVSLWLLYILFATLEAYCYIKGF",
# SNX16_HUMAN
"MATPYVPVPMPIGNSASSFTTNRNQRSSSFGSVSTSSNSSKGQLEDSNMGNFKQTSVPDQMDNTSSVCSSPLIRTKFTGTASSIEYSTRPRDTEEQNPETVNWEDRPSTPTILGYEVMEERAKFTVYKILVKKTPEESWVVFRRYTDFSRLNDKLKEMFPGFRLALPPKRWFKDNYNADFLEDRQLGLQAFLQNLVAHKDIANCLAVREFLCLDDPPGPFDSLEESRAFCETLEETNYRLQKELLEKQKEMESLKKLLSEKQLHIDTLENRIRTLSLEPEESLDVSETEGEQILKVESSALEVDQDVLDEESRADNKPCLSFSEPENAVSEIEVAEVAYDAEED",
# NRX1B_HUMAN
"MYQRMLRCGAELGSPGGGGGGGGGGGAGGRLALLWIVPLTLSGLLGVAWGASSLGAHHIHHFHGSSKHHSVPIAIYRSPASLRGGHAGTTYIFSKGGGQITYKWPPNDRPSTRADRLAIGFSTVQKEAVLVRVDSSSGLGDYLELHIHQGKIGVKFNVGTDDIAIEESNAIINDGKYHVVRFTRSGGNATLQVDSWPVIERYPAGNNDNERLAIARQRIPYRLGRVVDEWLLDKGRQLTIFNSQATIIIGGKEQGQPFQGQLSGLYYNGLKVLNMAAENDANIAIVGNVRLVGEVPSSMTTESTATAMQSEMSTSIMETTTTLATSTARRGKPPTKEPISQTTDDILVASAECPSDDEDIDPCEPSSGGLANPTRAGGREPYPGSAEVIRESSSTTGMVVGIVAAAALCILILLYAMYKYRNRDEGSYHVDESRNYISNSAQSNGAVVKEKQPSSAKSSNKNKKNKDKEYYV",
# NRX2B_HUMAN
"MPPGGSGPGGCPRRPPALAGPLPPPPPPPPPPLLPLLPLLLLLLLGAAEGARVSSSLSTTHHVHHFHSKHGTVPIAINRMPFLTRGGHAGTTYIFGKGGALITYTWPPNDRPSTRMDRLAVGFSTHQRSAVLVRVDSASGLGDYLQLHIDQGTVGVIFNVGTDDITIDEPNAIVSDGKYHVVRFTRSGGNATLQVDSWPVNERYPAGNFDNERLAIARQRIPYRLGRVVDEWLLDKGRQLTIFNSQAAIKIGGRDQGRPFQGQVSGLYYNGLKVLALAAESDPNVRTEGHLRLVGEGPSVLLSAETTATTLLADMATTIMETTTTMATTTTRRGRSPTLRDSTTQNTDDLLVASAECPSDDEDLEECEPSTGGELILPIITEDSLDPPPVATRSPFVPPPPTFYPFLTGVGATQDTLPPPAARRPPSGGPCQAERDDSDCEEPIEASGFASGEVFDSSLPPTDDEDFYTTFPLVTDRTTLLSPRKPAPRPNLRTDGATGAPGVLFAPSAPAPNLPAGKMNHRDPLQPLLENPPLGPGAPTSFEPRRPPPLRPGVTSAPGFPHLPTANPTGPGERGPPGAVEVIRESSSTTGMVVGIVAAAALCILILLYAMYKYRNRDEGSYQVDQSRNYISNSAQSNGAVVKEKAPAAPKTPSKAKKNKDKEYYV",
# NXPH1_HUMAN
"MQAACWYVLFLLQPTVYLVTCANLTNGGKSELLKSGSSKSTLKHIWTESSKDLSISRLLSQTFRGKENDTDLDLRYDTPEPYSEQDLWDWLRNSTDLQEPRPRAKRRPIVKTGKFKKMFGWGDFHSNIKTVKLNLLITGKIVDHGNGTFSVYFRHNSTGQGNVSVSLVPPTKIVEFDLAQQTVIDAKDSKSFNCRIEYEKVDKATKNTLCNYDPSKTCYQEQTQSHVSWLCSKPFKVICIYISFYSTDYKLVQKVCPDYNYHSDTPYFPSG",
# GBG2_HUMAN
"MASNNTASIAQARKLVEQLKMEANIDRIKVSKAAADLMAYCEAHAKEDPLLTPVPASENPFREKKFFCAIL",
# AP3S2_HUMAN
"MIQAILVFNNHGKPRLVRFYQRFPEEIQQQIVRETFHLVLKRDDNICNFLEGGSLIGGSDYKLIYRHYATLYFVFCVDSSESELGILDLIQVFVETLDKCFENVCELDLIFHMDKVHYILQEVVMGGMVLETNMNEIVAQIEAQNRLEKSEGGLSAAPARAVSAVKNINLPEIPRNINIGDLNIKVPNLSQFV",
# EIF3E_HUMAN
"MAEYDLTTRIAHFLDRHLVFPLLEFLSVKEIYNEKELLQGKLDLLSDTNMVDFAMDVYKNLYSDDIPHALREKRTTVVAQLKQLQAETEPIVKMFEDPETTRQMQSTRDGRMLFDYLADKHGFRQEYLDTLYRYAKFQYECGNYSGAAEYLYFFRVLVPATDRNALSSLWGKLASEILMQNWDAAMEDLTRLKETIDNNSVSSPLQSLQQRTWLIHWSLFVFFNHPKGRDNIIDLFLYQPQYLNAIQTMCPHILRYLTTAVITNKDVRKRRQVLKDLVKVIQQESYTYKDPITEFVECLYVNFDFDGAQKKLRECESVLVNDFFLVACLEDFIENARLFIFETFCRIHQCISINMLADKLNMTPEEAERWIVNLIRNARLDAKIDSKLGHVVMGNNAVSPYQQVIEKTKSLSFRSQMLAMNIEKKLNQNSRSEAPNWATQDSGFY",
# PTEN_HUMAN
"MTAIIKEIVSRNKRRYQEDGFDLDLTYIYPNIIAMGFPAERLEGVYRNNIDDVVRFLDSKHKNHYKIYNLCAERHYDTAKFNCRVAQYPFEDHNPPQLELIKPFCEDLDQWLSEDDNHVAAIHCKAGKGRTGVMICAYLLHRGKFLKAQEALDFYGEVRTRDKKGVTIPSQRRYVYYYSYLLKNHLDYRPVALLFHKMMFETIPMFSGGTCNPQFVVCQLKVKIYSSNSGPTRREDKFMYFEFPQPLPVCGDIKVEFFHKQNKMLKKDKMFHFWVNTFFIPGPEETSEKVENGSLCDQEIDSICSIERADNDKEYLVLTLTKNDLDKANKDKANRYFSPNFKVKLYFTKTVEEPSNPEASSSTSVTPDVSDNEPDHYRYSDTTDSDPENEPFDEDQHTQITKV",
# ACTB_HUMAN
"MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVGDEAQSKRGILTLKYPIEHGIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETFNTPAMYVAIQAVLSLYASGRTTGIVMDSGDGVTHTVPIYEGYALPHAILRLDLAGRDLTDYLMKILTERGYSFTTTAEREIVRDIKEKLCYVALDFEQEMATAASSSSLEKSYELPDGQVITIGNERFRCPEALFQPSFLGMESCGIHETTFNSIMKCDVDIRKDLYANTVLSGGTTMYPGIADRMQKEITALAPSTMKIKIIAPPERKYSVWIGGSILASLSTFQQMWISKQEYDESGPSIVHRKCF",
# RAC3_HUMAN
"MQAIKCVVVGDGAVGKTCLLISYTTNAFPGEYIPTVFDNYSANVMVDGKPVNLGLWDTAGQEDYDRLRPLSYPQTDVFLICFSLVSPASFENVRAKWYPEVRHHCPHTPILLVGTKLDLRDDKDTIERLRDKKLAPITYPQGLAMAREIGSVKYLECSALTQRGLKTVFDEAIRAVLCPPPVKKPGKKCTVF",
# RS20_HUMAN
"MAFKDTGKTPVEPEVAIHRIRITLTSRNVKSLEKVCADLIRGAKEKNLKVKGPVRMPTKTLRITTRKTPCGEGSKTWDRFQMRIHKRLIDLHSPSEIVKQITSISIEPGVEVEVTIADA",
# SNP25_HUMAN
"MAEDADMRNELEEMQRRADQLADESLESTRRMLQLVEESKDAGIRTLVMLDEQGEQLERIEEGMDQINKDMKEAEKNLTDLGKFCGLCVCPCNKLKSSDAYKKAWGNNQDGVVASQPARVVDEREQMAISGGFIRRVTNDARENEMDENLEQVSGIIGNLRHMALDMGNEIDTQNRQIDRIMEKADSNKTRIDEANQRATKMLGSG",
# CDC42_HUMAN
"MQTIKCVVVGDGAVGKTCLLISYTTNKFPSEYVPTVFDNYAVTVMIGGEPYTLGLFDTAGQEDYDRLRPLSYPQTDVFLVCFSVVSPSSFENVKEKWVPEITHHCPKTPFLLVGTQIDLRDDPSTIEKLAKNKQKPITPETAEKLARDLKAVKYVECSALTQKGLKNVFDEAILAALEPPEPKKSRRCVLL",
# DEST_HUMAN
"MASGVQVADEVCRIFYDMKVRKCSTPEEIKKRKKAVIFCLSADKKCIIVEEGKEILVGDVGVTITDPFKHFVGMLPEKDCRYALYDASFETKESRKEELMFFLWAPELAPLKSKMIYASSKDAIKKKFQGIKHECQANGPEDLNRACIAEKLGGSLIVAFEGCPV",
# RAB8A_HUMAN
"MAKTYDYLFKLLLIGDSGVGKTCVLFRFSEDAFNSTFISTIGIDFKIRTIELDGKRIKLQIWDTAGQERFRTITTAYYRGAMGIMLVYDITNEKSFDNIRNWIRNIEEHASADVEKMILGNKCDVNDKRQVSKERGEKLALDYGIKFMETSAKANINVENAFFTLARDIKAKMDKKLEGNSPQGSNQGVKITPDQQKRSSFFRCVLL",
# RAB4B_HUMAN
"MAETYDFLFKFLVIGSAGTGKSCLLHQFIENKFKQDSNHTIGVEFGSRVVNVGGKTVKLQIWDTAGQERFRSVTRSYYRGAAGALLVYDITSRETYNSLAAWLTDARTLASPNIVVILCGNKKDLDPEREVTFLEASRFAQENELMFLETSALTGENVEEAFLKCARTILNKIDSGELDPERMGSGIQYGDASLRQLRQPRSAQAVAPQPCGC",
# RAB2A_HUMAN
"MAYAYLFKYIIIGDTGVGKSCLLLQFTDKRFQPVHDLTIGVEFGARMITIDGKQIKLQIWDTAGQESFRSITRSYYRGAAGALLVYDITRRDTFNHLTTWLEDARQHSNSNMVIMLIGNKSDLESRREVKKEEGEAFAREHGLIFMETSAKTASNVEEAFINTAKEIYEKIQEGVFDINNEANGIKIGPQHAATNATHAGNQGGQQAGGGCC",
# RAB5B_HUMAN
"MTSRSTARPNGQPQASKICQFKLVLLGESAVGKSSLVLRFVKGQFHEYQESTIGAAFLTQSVCLDDTTVKFEIWDTAGQERYHSLAPMYYRGAQAAIVVYDITNQETFARAKTWVKELQRQASPSIVIALAGNKADLANKRMVEYEEAQAYADDNSLLFMETSAKTAMNVNDLFLAIAKKLPKSEPQNLGGAAGRSRGVDLHEQSQQNKSQCCSN",
# RAB10_HUMAN
"MAKKTYDLLFKLLLIGDSGVGKTCVLFRFSDDAFNTTFISTIGIDFKIKTVELQGKKIKLQIWDTAGQERFHTITTSYYRGAMGIMLVYDITNGKSFENISKWLRNIDEHANEDVERMLLGNKCDMDDKRVVPKGKGEQIAREHGIRFFETSAKANINIEKAFLTLAEDILRKTPVKEPNSENVDISSGGGVTGWKSKCC",
# UBC12_HUMAN
"MIKLFSLKQQKKEEESAGGTKGSSKKASAAQLRIQKDINELNLPKTCDISFSDPDDLLNFKLVICPDEGFYKSGKFVFSFKVGQGYPHDPPKVKCETMVYHPNIDLEGNVCLNILREDWKPVLTINSIIYGLQYLFLEPNPEDPLNKEAAEVLQNNRRLFEQNVQRSMRGGYIGSTYFERCLK",
# UBE2N_HUMAN
"MAGLPRRIIKETQRLLAEPVPGIKAEPDESNARYFHVVIAGPQDSPFEGGTFKLELFLPEEYPMAAPKVRFMTKIYHPNVDKLGRICLDILKDKWSPALQIRTVLLSIQALLSAPNPDDPLANDVAEQWKTNEAQAIETARAWTRLYAMNNI",
# RAB14_HUMAN
"MATAPYNYSYIFKYIIIGDMGVGKSCLLHQFTEKKFMADCPHTIGVEFGTRIIEVSGQKIKLQIWDTAGQERFRAVTRSYYRGAAGALMVYDITRRSTYNHLSSWLTDARNLTNPNTVIILIGNKADLEAQRDVTYEEAKQFAEENGLLFLEASAKTGENVEDAFLEAAKKIYQNIQDGSLDLNAAESGVQHKPSAPQGGRLTSEPQPQREGCGC",
# ARP3_HUMAN
"MAGRLPACVVDCGTGYTKLGYAGNTEPQFIIPSCIAIKESAKVGDQAQRRVMKGVDDLDFFIGDEAIEKPTYATKWPIRHGIVEDWDLMERFMEQVIFKYLRAEPEDHYFLLTEPPLNTPENREYTAEIMFESFNVPGLYIAVQAVLALAASWTSRQVGERTLTGTVIDSGDGVTHVIPVAEGYVIGSCIKHIPIAGRDITYFIQQLLRDREVGIPPEQSLETAKAVKERYSYVCPDLVKEFNKYDTDGSKWIKQYTGINAISKKEFSIDVGYERFLGPEIFFHPEFANPDFTQPISEVVDEVIQNCPIDVRRPLYKNIVLSGGSTMFRDFGRRLQRDLKRTVDARLKLSEELSGGRLKPKPIDVQVITHHMQRYAVWFGGSMLASTPEFYQVCHTKKDYEEIGPSICRHNPVFGVMS",
# ARP2_HUMAN
"MDSQGRKVVVCDNGTGFVKCGYAGSNFPEHIFPALVGRPIIRSTTKVGNIEIKDLMVGDEASELRSMLEVNYPMENGIVRNWDDMKHLWDYTFGPEKLNIDTRNCKILLTEPPMNPTKNREKIVEVMFETYQFSGVYVAIQAVLTLYAQGLLTGVVVDSGDGVTHICPVYEGFSLPHLTRRLDIAGRDITRYLIKLLLLRGYAFNHSADFETVRMIKEKLCYVGYNIEQEQKLALETTVLVESYTLPDGRIIKVGGERFEAPEALFQPHLINVEGVGVAELLFNTIQAADIDTRSEFYKHIVLSGGSTMYPGLPSRLERELKQLYLERVLKGDVEKLSKFKIRIEDPPRRKHMVFLGGAVLADIMKDKDNFWMTRQEYQEKGVRVLEKLGVTVR",
# RAP1B_HUMAN
"MREYKLVVLGSGGVGKSALTVQFVQGIFVEKYDPTIEDSYRKQVEVDAQQCMLEILDTAGTEQFTAMRDLYMKNGQGFALVYSITAQSTFNDLQDLREQILRVKDTDDVPMILVGNKCDLEDERVVGKEQGQNLARQWNNCAFLESSAKSKINVNEIFYDLVRQINRKTPVPGKARKKSSCQLL",
# RS3A_HUMAN
"MAVGKNKRLTKGGKKGAKKKVVDPFSKKDWYDVKAPAMFNIRNIGKTLVTRTQGTKIASDGLKGRVFEVSLADLQNDEVAFRKFKLITEDVQGKNCLTNFHGMDLTRDKMCSMVKKWQTMIEAHVDVKTTDGYLLRLFCVGFTKKRNNQIRKTSYAQHQQVRQIRKKMMEIMTREVQTNDLKEVVNKLIPDSIGKDIEKACQSIYPLHDVFVRKVKMLKKPKFELGKLMELHGEGSSSGKATGDETGAKVERADGYEPPVQESV",
# RL26_HUMAN
"MKFNPFVTSDRSKNRKRHFNAPSHIRRKIMSSPLSKELRQKYNVRSMPIRKDDEVQVVRGHYKGQQIGKVVQVYRKKYVIYIERVQREKANGTTVHVGIHPSKVVITRLKLDKDRKKILERKAKSRQVGKEKGKYKEETIEKMQE",
# STX1B_HUMAN
"MKDRTQELRSAKDSDDEEEVVHVDRDHFMDEFFEQVEEIRGCIEKLSEDVEQVKKQHSAILAAPNPDEKTKQELEDLTADIKKTANKVRSKLKAIEQSIEQEEGLNRSSADLRIRKTQHSTLSRKFVEVMTEYNATQSKYRDRCKDRIQRQLEITGRTTTNEELEDMLESGKLAIFTDDIKMDSQMTKQALNEIETRHNEIIKLETSIRELHDMFVDMAMLVESQGEMIDRIEYNVEHSVDYVERAVSDTKKAVKYQSKARRKKIMIIICCVVLGVVLASSIGGTLGL",
# SMS_HUMAN
"MLSCRLQCALAALSIVLALGCVTGAPSDPRLRQFLQKSLAAAAGKQELAKYFLAELLSEPNQTENDALEPEDLSQAAEQDEMRLELQRSANSNPAMAPRERKAGCKNFFWKTFTSC",
# RL15_HUMAN
"MGAYKYIQELWRKKQSDVMRFLLRVRCWQYRQLSALHRAPRPTRPDKARRLGYKAKQGYVIYRIRVRRGGRKRPVPKGATYGKPVHHGVNQLKFARSLQSVAEERAGRHCGALRVLNSYWVGEDSTYKFFEVILIDPFHKAIRRNPDTQWITKPVHKHREMRGLTSAGRKSRGLGKGHKFHHTIGGSRRAAWRRRNTLQLHRYR",
# RL27_HUMAN
"MGKFMKPGKVVLVLAGRYSGRKAVIVKNIDDGTSDRPYSHALVAGIDRYPRKVTAAMGKKKIAKRSKIKSFVKVYNYNHLMPTRYSVDIPLDKTVVNKDVFRDPALKRKARREAKVKFEERYKTGKNKWFFQKLRF",
# VA0D1_HUMAN
"MSFFPELYFNVDNGYLEGLVRGLKAGVLSQADYLNLVQCETLEDLKLHLQSTDYGNFLANEASPLTVSVIDDRLKEKMVVEFRHMRNHAYEPLASFLDFITYSYMIDNVILLITGTLHQRSIAELVPKCHPLGSFEQMEAVNIAQTPAELYNAILVDTPLAAFFQDCISEQDLDEMNIEIIRNTLYKAYLESFYKFCTLLGGTTADAMCPILEFEADRRAFIITINSFGTELSKEDRAKLFPHCGRLYPEGLAQLARADDYEQVKNVADYYPEYKLLFEGAGSNPGDKTLEDRFFEHEVKLNKLAFLNQFHFGVFYAFVKLKEQECRNIVWIAECIAQRHRAKIDNYIPIF",
# RL37A_HUMAN
"MAKRTKKVGIVGKYGTRYGASLRKMVKKIEISQHAKYTCSFCGKTKMKRRAVGIWHCGSCMKTVAGGAWTYNTTSAVTVKSAIRRLKELKDQ",
# RHOA_HUMAN
"MAAIRKKLVIVGDGACGKTCLLIVFSKDQFPEVYVPTVFENYVADIEVDGKQVELALWDTAGQEDYDRLRPLSYPDTDVILMCFSIDSPDSLENIPEKWTPEVKHFCPNVPIILVGNKKDLRNDEHTRRELAKMKQEPVKPEEGRDMANRIGAFGYMECSAKTKDGVREVFEMATRAALQARRGKKKSGCLVL",
# STXB1_HUMAN
"MAPIGLKAVVGEKIMHDVIKKVKKKGEWKVLVVDQLSMRMLSSCCKMTDIMTEGITIVEDINKRREPLPSLEAVYLITPSEKSVHSLISDFKDPPTAKYRAAHVFFTDSCPDALFNELVKSRAAKVIKTLTEINIAFLPYESQVYSLDSADSFQSFYSPHKAQMKNPILERLAEQIATLCATLKEYPAVRYRGEYKDNALLAQLIQDKLDAYKADDPTMGEGPDKARSQLLILDRGFDPSSPVLHELTFQAMSYDLLPIENDVYKYETSGIGEARVKEVLLDEDDDLWIALRHKHIAEVSQEVTRSLKDFSSSKRMNTGEKTTMRDLSQMLKKMPQYQKELSKYSTHLHLAEDCMKHYQGTVDKLCRVEQDLAMGTDAEGEKIKDPMRAIVPILLDANVSTYDKIRIILLYIFLKNGITEENLNKLIQHAQIPPEDSEIITNMAHLGVPIVTDSTLRRRSKPERKERISEQTYQLSRWTPIIKDIMEDTIEDKLDTKHYPYISTRSSASFSTTAVSARYGHWHKNKAPGEYRSGPRLIIFILGGVSLNEMRCAYEVTQANGKWEVLIGSTHILTPQKLLDTLKKLNKTDEEISS",
# RL37_HUMAN
"MTKGTSSFGKRRNKTHTLCRRCGSKAYHLQKSTCGKCGYPAKRKRKYNWSAKAKRRNTTGTGRMRHLKIVYRRFRHGFREGTTPKPKRAAVAASSSS",
# SUMO2_HUMAN
"MADEKPKEGVKTENNDHINLKVAGQDGSVVQFKIKRHTPLSKLMKAYCERQGLSMRQIRFRFDGQPINETDTPAQLEMEDEDTIDVFQQQTGGVY",
# AP1S1_HUMAN
"MMRFMLLFSRQGKLRLQKWYLATSDKERKKMVRELMQVVLARKPKMCSFLEWRDLKVVYKRYASLYFCCAIEGQDNELITLELIHRYVELLDKYFGSVCELDIIFNFEKAYFILDEFLMGGDVQDTSKKSVLKAIEQADLLQEEDESPRSVLEEMGLA",
# HNRPK_HUMAN
"METEQPEETFPNTETNGEFGKRPAEDMEEEQAFKRSRNTDEMVELRILLQSKNAGAVIGKGGKNIKALRTDYNASVSVPDSSGPERILSISADIETIGEILKKIIPTLEEGLQLPSPTATSQLPLESDAVECLNYQHYKGSDFDCELRLLIHQSLAGGIIGVKGAKIKELRENTQTTIKLFQECCPHSTDRVVLIGGKPDRVVECIKIILDLISESPIKGRAQPYDPNFYDETYDYGGFTMMFDDRRGRPVGFPMRGRGGFDRMPPGRGGRPMPPSRRDYDDMSPRRGPPPPPPGRGGRGGSRARNLPLPPPPPPRGGDLMAYDRRGRPGDRYDGMVGFSADETWDSAIDTWSPSEWQMAYEPQGGSGYDYSYAGGRGSYGDLGGPIITTQVTIPKDLAGSIIGKGGQRIKQIRHESGASIKIDEPLEGSEDRIITITGTQDQIQNAQYLLQNSVKQYSGKFF",
# 1433G_HUMAN
"MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLSVAYKNVVGARRSSWRVISSIEQKTSADGNEKKIEMVRAYREKIEKELEAVCQDVLSLLDNYLIKNCSETQYESKVFYLKMKGDYYRYLAEVATGEKRATVVESSEKAYSEAHEISKEHMQPTHPIRLGLALNYSVFYYEIQNAPEQACHLAKTAFDDAIAELDTLNEDSYKDSTLIMQLLRDNLTLWTSDQQDDDGGEGNN",
# UBP46_HUMAN
"MTVRNIASICNMGTNASALEKDIGPEQFPINEHYFGLVNFGNTCYCNSVLQALYFCRPFRENVLAYKAQQKKKENLLTCLADLFHSIATQKKKVGVIPPKKFISRLRKENDLFDNYMQQDAHEFLNYLLNTIADILQEEKKQEKQNGKLKNGNMNEPAENNKPELTWVHEIFQGTLTNETRCLNCETVSSKDEDFLDLSVDVEQNTSITHCLRDFSNTETLCSEQKYYCETCCSKQEAQKRMRVKKLPMILALHLKRFKYMEQLHRYTKLSYRVVFPLELRLFNTSSDAVNLDRMYDLVAVVVHCGSGPNRGHYITIVKSHGFWLLFDDDIVEKIDAQAIEEFYGLTSDISKNSESGYILFYQSRE",
# RS7_HUMAN
"MFSSSAKIVKPNGEKPDEFESGISQALLELEMNSDLKAQLRELNITAAKEIEVGGGRKAIIIFVPVPQLKSFQKIQVRLVRELEKKFSGKHVVFIAQRRILPKPTRKSRTKNKQKRPRSRTLTAVHDAILEDLVFPSEIVGKRIRVKLDGSRLIKVHLDKAQQNNVEHKVETFSGVYKKLTGKDVNFEFPEFQL",
# PP1A_HUMAN
"MSDSEKLNLDSIIGRLLEVQGSRPGKNVQLTENEIRGLCLKSREIFLSQPILLELEAPLKICGDIHGQYYDLLRLFEYGGFPPESNYLFLGDYVDRGKQSLETICLLLAYKIKYPENFFLLRGNHECASINRIYGFYDECKRRYNIKLWKTFTDCFNCLPIAAIVDEKIFCCHGGLSPDLQSMEQIRRIMRPTDVPDQGLLCDLLWSDPDKDVQGWGENDRGVSFTFGAEVVAKFLHKHDLDLICRAHQVVEDGYEFFAKRQLVTLFSAPNYCGEFDNAGAMMSVDETLMCSFQILKPADKNKGKYGQFSGLNPGGRPITPPRNSAKAKK",
# NCS1_HUMAN
"MGKSNSKLKPEVVEELTRKTYFTEKEVQQWYKGFIKDCPSGQLDAAGFQKIYKQFFPFGDPTKFATFVFNVFDENKDGRIEFSEFIQALSVTSRGTLDEKLRWAFKLYDLDNDGYITRNEMLDIVDAIYQMVGNTVELPEEENTPEKRVDRIFAMMDKNADGKLTLQEFQEGSKADPSIVQALSLYDGLV",
# RS8_HUMAN
"MGISRDNWHKRRKTGGKRKPYHKKRKYELGRPAANTKIGPRRIHTVRVRGGNKKYRALRLDVGNFSWGSECCTRKTRIIDVVYNASNNELVRTKTLVKNCIVLIDSTPYRQWYESHYALPLGRKKGAKLTPEEEEILNKKRSKKIQKKYDERKKNAKISSLLEEQFQQGKLLACIASRPGQCGRADGYVLEGKELEFYLRKIKARKGK",
# RS15A_HUMAN
"MVRMNVLADALKSINNAEKRGKRQVLIRPCSKVIVRFLTVMMKHGYIGEFEIIDDHRAGKIVVNLTGRLNKCGVISPRFDVQLKDLEKWQNNLLPSRQFGFIVLTTSAGIMDHEEARRKHTGGKILGFFF",
# RS16_HUMAN
"MPSKGPLQSVQVFGRKKTATAVAHCKRGNGLIKVNGRPLEMIEPRTLQYKLLEPVLLLGKERFAGVDIRVRVKGGGHVAQIYAIRQSISKALVAYYQKYVDEASKKEIKDILIQYDRTLLVADPRRCESKKFGGPGARARYQKSYR",
# 1433E_HUMAN
"MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLSVAYKNVIGARRASWRIISSIEQKEENKGGEDKLKMIREYRQMVETELKLICCDILDVLDKHLIPAANTGESKVFYYKMKGDYHRYLAEFATGNDRKEAAENSLVAYKAASDIAMTELPPTHPIRLGLALNFSVFYYEILNSPDRACRLAKAAFDDAIAELDTLSEESYKDSTLIMQLLRDNLTLWTSDMQGDGEEQNKEALQDVEDENQ",
# RS14_HUMAN
"MAPRKGKEKKEEQVISLGPQVAEGENVFGVCHIFASFNDTFVHVTDLSGKETICRVTGGMKVKADRDESSPYAAMLAAQDVAQRCKELGITALHIKLRATGGNRTKTPGPGAQSALRALARSGMKIGRIEDVTPIPSDSTRRKGGRRGRRL",
# RS23_HUMAN
"MGKCRGLRTARKLRSHRRDQKWHDKQYKKAHLGTALKANPFGGASHAKGIVLEKVGVEAKQPNSAIRKCVRVQLIKNGKKITAFVPNDGCLNFIEENDEVLVAGFGRKGHAVGDIPGVRFKVVKVANVSLLALYKGKKERPRS",
# RS18_HUMAN
"MSLVIPEKFQHILRVLNTNIDGRRKIAFAITAIKGVGRRYAHVVLRKADIDLTKRAGELTEDEVERVITIMQNPRQYKIPDWFLNRQKDVKDGKYSQVLANGLDNKLREDLERLKKIRAHRGLRHFWGLRVRGQHTKTTGRRGRTVGVSKKK",
# RS13_HUMAN
"MGRMHAPGKGLSQSALPYRRSVPTWLKLTSDDVKEQIYKLAKKGLTPSQIGVILRDSHGVAQVRFVTGNKILRILKSKGLAPDLPEDLYHLIKKAVAVRKHLERNRKDKDAKFRLILIESRIHRLARYYKTKRVLPPNWKYESSTASALVA",
# RS11_HUMAN
"MADIQTERAYQKQPTIFQNKKRVLLGETGKEKLPRYYKNIGLGFKTPKEAIEGTYIDKKCPFTGNVSIRGRILSGVVTKMKMQRTIVIRRDYLHYIRKYNRFEKRHKNMSVHLSPCFRDVQIGDIVTVGECRPLSKTVRFNVLKVTKAAGTKKQFQKF",
# ARF6_HUMAN
"MGKVLSKIFGNKEMRILMLGLDAAGKTTILYKLKLGQSVTTIPTVGFNVETVTYKNVKFNVWDVGGQDKIRPLWRHYYTGTQGLIFVVDCADRDRIDEARQELHRIINDREMRDAIILIFANKQDLPDAMKPHEIQEKLGLTRIRDRNWYVQPSCATSGDGLYEGLTWLTSNYKS",
# RL7A_HUMAN
"MPKGKKAKGKKVAPAPAVVKKQEAKKVVNPLFEKRPKNFGIGQDIQPKRDLTRFVKWPRYIRLQRQRAILYKRLKVPPAINQFTQALDRQTATQLLKLAHKYRPETKQEKKQRLLARAEKKAAGKGDVPTKRPPVLRAGVNTVTTLVENKKAQLVVIAHDVDPIELVVFLPALCRKMGVPYCIIKGKARLGRLVHRKTCTTVAFTQVNSEDKGALAKLVEAIRTNYNDRYDEIRRHWGGNVLGPKSVARIAKLEKAKAKELATKLG",
# RB11A_HUMAN
"MGTRDDEYDYLFKVVLIGDSGVGKSNLLSRFTRNEFNLESKSTIGVEFATRSIQVDGKTIKAQIWDTAGQERYRAITSAYYRGAVGALLVYDIAKHLTYENVERWLKELRDHADSNIVIMLVGNKSDLRHLRAVPTDEARAFAEKNGLSFIETSALDSTNVEAAFQTILTEIYRIVSQKQMSDRRENDMSPSNNVVPIHVPPTTENKPKVQCCQNI",
# RS4X_HUMAN
"MARGPKKHLKRVAAPKHWMLDKLTGVFAPRPSTGPHKLRECLPLIIFLRNRLKYALTGDEVKKICMQRFIKIDGKVRTDITYPAGFMDVISIDKTGENFRLIYDTKGRFAVHRITPEEAKYKLCKVRKIFVGTKGIPHLVTHDARTIRYPDPLIKVNDTIQIDLETGKITDFIKFDTGNLCMVTGGANLGRIGVITNRERHPGSFDVVHVKDANGNSFATRLSNIFVIGKGNKPWISLPRGKGIRLTIAEERDKRLAAKQSSG",
# RHOB_HUMAN
"MAAIRKKLVVVGDGACGKTCLLIVFSKDEFPEVYVPTVFENYVADIEVDGKQVELALWDTAGQEDYDRLRPLSYPDTDVILMCFSVDSPDSLENIPEKWVPEVKHFCPNVPIILVANKKDLRSDEHVRTELARMKQEPVRTDDGRAMAVRIQAYDYLECSAKTKEGVREVFETATRAALQKRYGSQNGCINCCKVL",
# RL23A_HUMAN
"MAPKAKKEAPAPPKAEAKAKALKAKKAVLKGVHSHKKKKIRTSPTFRRPKTLRLRRQPKYPRKSAPRRNKLDHYAIIKFPLTTESAMKKIEDNNTLVFIVDVKANKHQIKQAVKKLYDIDVAKVNTLIRPDGEKKAYVRLAPDYDALDVANKIGII",
# RS6_HUMAN
"MKLNISFPATGCQKLIEVDDERKLRTFYEKRMATEVAADALGEEWKGYVVRISGGNDKQGFPMKQGVLTHGRVRLLLSKGHSCYRPRRTGERKRKSVRGCIVDANLSVLNLVIVKKGEKDIPGLTDTTVPRRLGPKRASRIRKLFNLSKEDDVRQYVVRKPLNKEGKKPRTKAPKIQRLVTPRVLQHKRRRIALKKQRTKKNKEEAAEYAKLLAKRMKEAKEKRQEQIAKRRRLSSLRASTSKSESSQK",
# RAB1A_HUMAN
"MSSMNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVDFKIRTIELDGKTIKLQIWDTAGQERFRTITSSYYRGAHGIIVVYDVTDQESFNNVKQWLQEIDRYASENVNKLLVGNKCDLTTKKVVDYTTAKEFADSLGIPFLETSAKNATNVEQSFMTMAAEIKKRMGPGATAGGAEKSNVKIQSTPVKQSGGGCC",
# RL23_HUMAN
"MSKRGRGGSSGAKFRISLGLPVGAVINCADNTGAKNLYIISVKGIKGRLNRLPAAGVGDMVMATVKKGKPELRKKVHPAVVIRQRKSYRRKDGVFLYFEDNAGVIVNNKGEMKGSAITGPVAKECADLWPRIASNAGSIA",
# RAP1A_HUMAN
"MREYKLVVLGSGGVGKSALTVQFVQGIFVEKYDPTIEDSYRKQVEVDCQQCMLEILDTAGTEQFTAMRDLYMKNGQGFALVYSITAQSTFNDLQDLREQILRVKDTEDVPMILVGNKCDLEDERVVGKEQGQNLARQWCNCAFLESSAKSKINVNEIFYDLVRQINRKTPVEKKKPKKKSCLLL",
# RS15_HUMAN
"MAEVEQKKKRTFRKFTYRGVDLDQLLDMSYEQLMQLYSARQRRRLNRGLRRKQHSLLKRLRKAKKEAPPMEKPEVVKTHLRDMIILPEMVGSMVGVYNGKTFNQVEIKPEMIGHYLGEFSITYKPVKHGRPGIGATHSSRFIPLK",
# RS24_HUMAN
"MNDTVTIRTRKFMTNRLLQRKQMVIDVLHPGKATVPKTEIREKLAKMYKTTPDVIFVFGFRTHFGGGKTTGFGMIYDSLDYAKKNEPKHRLARHGLYEKKKTSRKQRKERKNRMKKVRGTAKANVGAGKKPKE",
# RS25_HUMAN
"MPPKDDKKKKDAGKSAKKDKDPVNKSGGKAKKKKWSKGKVRDKLNNLVLFDKATYDKLCKEVPNYKLITPAVVSERLKIRGSLARAALQELLSKGLIKLVSKHRAQVIYTRNTKGGDAPAAGEDA",
# RS26_HUMAN
"MTKKRRNNGRAKKGRGHVQPIRCTNCARCVPKDKAIKKFVIRNIVEAAAVRDISEASVFDAYVLPKLYVKLHYCVSCAIHSKVVRNRSREARKDRTPPPRFRPAGAAPRPPPKPM",
# RS28_HUMAN
"MDTSRVQPIKLARVTKVLGRTGSQGQCTQVRVEFMDDTSRSIIRNVKGPVREGDVLTLLESEREARRLR",
# GBB1_HUMAN
"MSELDQLRQEAEQLKNQIRDARKACADATLSQITNNIDPVGRIQMRTRRTLRGHLAKIYAMHWGTDSRLLVSASQDGKLIIWDSYTTNKVHAIPLRSSWVMTCAYAPSGNYVACGGLDNICSIYNLKTREGNVRVSRELAGHTGYLSCCRFLDDNQIVTSSGDTTCALWDIETGQQTTTFTGHTGDVMSLSLAPDTRLFVSGACDASAKLWDVREGMCRQTFTGHESDINAICFFPNGNAFATGSDDATCRLFDLRADQELMTYSHDNIICGITSVSFSKSGRLLLAGYDDFNCNVWDALKADRAGVLAGHDNRVSCLGVTDDGMAVATGSWDSFLKIWN",
# GBB2_HUMAN
"MSELEQLRQEAEQLRNQIRDARKACGDSTLTQITAGLDPVGRIQMRTRRTLRGHLAKIYAMHWGTDSRLLVSASQDGKLIIWDSYTTNKVHAIPLRSSWVMTCAYAPSGNFVACGGLDNICSIYSLKTREGNVRVSRELPGHTGYLSCCRFLDDNQIITSSGDTTCALWDIETGQQTVGFAGHSGDVMSLSLAPDGRTFVSGACDASIKLWDVRDSMCRQTFIGHESDINAVAFFPNGYAFTTGSDDATCRLFDLRADQELLMYSHDNIICGITSVAFSRSGRLLLAGYDDFNCNIWDAMKGDRAGVLAGHDNRVSCLGVTDDGMAVATGSWDSFLKIWN",
# RL30_HUMAN
"MVAAKKTKKSLESINSRLQLVMKSGKYVLGYKQTLKMIRQGKAKLVILANNCPALRKSEIEYYAMLAKTGVHHYSGNNIELGTACGKYYRVCTLAIIDPGDSDIIRSMPEQTGEK",
# RL31_HUMAN
"MAPAKKGGEKKKGRSAINEVVTREYTINIHKRIHGVGFKKRAPRALKEIRKFAMKEMGTPDVRIDTRLNKAVWAKGIRNVPYRIRVRLSRKRNEDEDSPNKLYTLVTYVPVTTFKNLQTVNVDEN",
# RL10A_HUMAN
"MSSKVSRDTLYEAVREVLHGNQRKRRKFLETVELQISLKNYDPQKDKRFSGTVRLKSTPRPKFSVCVLGDQQHCDEAKAVDIPHMDIEALKKLNKNKKLVKKLAKKYDAFLASESLIKQIPRILGPGLNKAGKFPSLLTHNENMVAKVDEVKSTIKFQMKKVLCLAVAVGHVKMTDDELVYNIHLAVNFLVSLLKKNWQNVRALYIKSTMGKPQRLY",
# RL32_HUMAN
"MAALRPLVKPKIVKKRTKKFIRHQSDRYVKIKRNWRKPRGIDNRVRRRFKGQILMPNIGYGSNKKTKHMLPSGFRKFLVHNVKELEVLLMCNKSYCAEIAHNVSSKNRKAIVERAAQLAIRVTNPNARLRSEENE",
# RL11_HUMAN
"MAQDQGEKENPMRELRIRKLCLNICVGESGDRLTRAAKVLEQLTGQTPVFSKARYTVRSFGIRRNEKIAVHCTVRGAKAEEILEKGLKVREYELRKNNFSDTGNFGFGIQEHIDLGIKYDPSIGIYGLDFYVVLGRPGFSIADKKRRTGCIGAKHRISKEEAMRWFQQKYDGIILPGK",
# RL8_HUMAN
"MGRVIRGQRKGAGSVFRAHVKHRKGAARLRAVDFAERHGYIKGIVKDIIHDPGRGAPLAKVVFRDPYRFKKRTELFIAAEGIHTGQFVYCGKKAQLNIGNVLPVGTMPEGTIVCCLEEKPGDRGKLARASGNYATVISHNPETKKTRVKLPSGSKKVISSANRAVVGVVAGGGRIDKPILKAGRAYHKYKAKRNCWPRVRGVAMNPVEHPFGGGNHQHIGKPSTIRRDAPAGRKVGLIAARRTGRLRGTKTVQEKEN",
# FKB1A_HUMAN
"MGVQVETISPGDGRTFPKRGQTCVVHYTGMLEDGKKFDSSRDRNKPFKFMLGKQEVIRGWEEGVAQMSVGQRAKLTISPDYAYGATGHPGIIPPHATLVFDVELLKLE",
# CCG7_HUMAN
"MSHCSSRALTLLSSVFGACGLLLVGIAVSTDYWLYMEEGTVLPQNQTTEVKMALHAGLWRVCFFAGREKGRCVASEYFLEPEINLVTENTENILKTVRTATPFPMVSLFLVFTAFVISNIGHIRPQRTILAFVSGIFFILSGLSLVVGLVLYISSINDEVMNRPSSSEQYFHYRYGWSFAFAASSFLLKEGAGVMSVYLFTKRYAEEEMYRPHPAFYRPRLSDCSDYSGQFLQPEAWRRGRSPSDISSDVSIQMTQNYPPAIKYPDHLHISTSPC",
# RS27A_HUMAN
"MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGAKKRKKKSYTTPKKNKHKRKKVKLAVLKYYKVDENGKISRLRRECPSDECGAGVFMASHFDRHYCGKCCLTYCFNKPEDK",
# RL40_HUMAN
"MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGIIEPSLRQLAQKYNCDKMICRKCYARLHPRAVNCRKKKCGHTNNLRPKKKVK",
# GRB2_HUMAN
"MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPHPWFFGKIPRAKAEEMLSKQRHDGAFLIRESESAPGDFSLSVKFGNDVQHFKVLRDGAGKYFLWVVKFNSLNELVDYHRSTSVSRNQQIFLRDIEQVPQQPTYVQALFDFDPQEDGELGFRRGDFIHVMDNSDPNWWKGACHGQTGMFPRNYVTPVNRNV",
# RAC1_HUMAN
"MQAIKCVVVGDGAVGKTCLLISYTTNAFPGEYIPTVFDNYSANVMVDGKPVNLGLWDTAGQEDYDRLRPLSYPQTDVFLICFSLVSPASFENVRAKWYPEVRHHCPNTPIILVGTKLDLRDDKDTIEKLKEKKLTPITYPQGLAMAKEIGAVKYLECSALTQRGLKTVFDEAIRAVLCPPPVKKRKRKCLLL",
# AP2B1_HUMAN
"MTDSKYFTTNKKGEIFELKAELNNEKKEKRKEAVKKVIAAMTVGKDVSSLFPDVVNCMQTDNLELKKLVYLYLMNYAKSQPDMAIMAVNSFVKDCEDPNPLIRALAVRTMGCIRVDKITEYLCEPLRKCLKDEDPYVRKTAAVCVAKLHDINAQMVEDQGFLDSLRDLIADSNPMVVANAVAALSEISESHPNSNLLDLNPQNINKLLTALNECTEWGQIFILDCLSNYNPKDDREAQSICERVTPRLSHANSAVVLSAVKVLMKFLELLPKDSDYYNMLLKKLAPPLVTLLSGEPEVQYVALRNINLIVQKRPEILKQEIKVFFVKYNDPIYVKLEKLDIMIRLASQANIAQVLAELKEYATEVDVDFVRKAVRAIGRCAIKVEQSAERCVSTLLDLIQTKVNYVVQEAIVVIRDIFRKYPNKYESIIATLCENLDSLDEPDARAAMIWIVGEYAERIDNADELLESFLEGFHDESTQVQLTLLTAIVKLFLKKPSETQELVQQVLSLATQDSDNPDLRDRGYIYWRLLSTDPVTAKEVVLSEKPLISEETDLIEPTLLDELICHIGSLASVYHKPPNAFVEGSHGIHRKHLPIHHGSTDAGDSPVGTTTATNLEQPQVIPSQGDLLGDLLNLDLGPPVNVPQVSSMQMGAVDLLGGGLDSLVGQSFIPSSVPATFAPSPTPAVVSSGLNDLFELSTGIGMAPGGYVAPKAVWLPAVKAKGLEISGTFTHRQGHIYMEMNFTNKALQHMTDFAIQFNKNSFGVIPSTPLAIHTPLMPNQSIDVSLPLNTLGPVMKMEPLNNLQVAVKNNIDVFYFSCLIPLNVLFVEDGKMERQVFLATWKDIPNENELQFQIKECHLNADTVSSKLQNNNVYTIAKRNVEGQDMLYQSLKLTNGIWILAELRIQPGNPNYTLSLKCRAPEVSQYIYQVYDSILKN",
# VAMP2_HUMAN
"MSATAATAPPAAPAGEGGPPAPPPNLTSNRRLQQTQAQVDEVVDIMRVNVDKVLERDQKLSELDDRADALQAGASQFETSAAKLKRKYWWKNLKMMIILGVICAIILIIIIVYFST",
# GNAS2_HUMAN
"MGCLGNSKTEDQRNEEKAQREANKKIEKQLQKDKQVYRATHRLLLLGAGESGKSTIVKQMRILHVNGFNGEGGEEDPQAARSNSDGEKATKVQDIKNNLKEAIETIVAAMSNLVPPVELANPENQFRVDYILSVMNVPDFDFPPEFYEHAKALWEDEGVRACYERSNEYQLIDCAQYFLDKIDVIKQADYVPSDQDLLRCRVLTSGIFETKFQVDKVNFHMFDVGGQRDERRKWIQCFNDVTAIIFVVASSSYNMVIREDNQTNRLQEALNLFKSIWNNRWLRTISVILFLNKQDLLAEKVLAGKSKIEDYFPEFARYTTPEDATPEPGEDPRVTRAKYFIRDEFLRISTASGDGRHYCYPHFTCAVDTENIRRVFNDCRDIIQRMHLRQYELL",
# GNAI1_HUMAN
"MGCTLSAEDKAAVERSKMIDRNLREDGEKAAREVKLLLLGAGESGKSTIVKQMKIIHEAGYSEEECKQYKAVVYSNTIQSIIAIIRAMGRLKIDFGDSARADDARQLFVLAGAAEEGFMTAELAGVIKRLWKDSGVQACFNRSREYQLNDSAAYYLNDLDRIAQPNYIPTQQDVLRTRVKTTGIVETHFTFKDLHFKMFDVGGQRSERKKWIHCFEGVTAIIFCVALSDYDLVLAEDEEMNRMHESMKLFDSICNNKWFTDTSIILFLNKKDLFEEKIKKSPLTICYPEYAGSNTYEEAAAYIQCQFEDLNKRKDTKEIYTHFTCATDTKNVQFVFDAVTDVIIKNNLKDCGLF",
# CANB1_HUMAN
"MGNEASYPLEMCSHFDADEIKRLGKRFKKLDLDNSGSLSVEEFMSLPELQQNPLVQRVIDIFDTDGNGEVDFKEFIEGVSQFSVKGDKEQKLRFAFRIYDMDKDGYISNGELFQVLKMMVGNNLKDTQLQQIVDKTIINADKDGDGRISFEEFCAVVGGLDIHKKMVVDV",
# 1433Z_HUMAN
"MDKNELVQKAKLAEQAERYDDMAACMKSVTEQGAELSNEERNLLSVAYKNVVGARRSSWRVVSSIEQKTEGAEKKQQMAREYREKIETELRDICNDVLSLLEKFLIPNASQAESKVFYLKMKGDYYRYLAEVAAGDDKKGIVDQSQQAYQEAFEISKKEMQPTHPIRLGLALNFSVFYYEILNSPEKACSLAKTAFDEAIAELDTLSEESYKDSTLIMQLLRDNLTLWTSDTQGDEAEAGEGGEN",
# 2ABA_HUMAN
"MAGAGGGNDIQWCFSQVKGAVDDDVAEADIISTVEFNHSGELLATGDKGGRVVIFQQEQENKIQSHSRGEYNVYSTFQSHEPEFDYLKSLEIEEKINKIRWLPQKNAAQFLLSTNDKTIKLWKISERDKRPEGYNLKEEDGRYRDPTTVTTLRVPVFRPMDLMVEASPRRIFANAHTYHINSISINSDYETYLSADDLRINLWHLEITDRSFNIVDIKPANMEELTEVITAAEFHPNSCNTFVYSSSKGTIRLCDMRASALCDRHSKLFEEPEDPSNRSFFSEIISSISDVKFSHSGRYMMTRDYLSVKIWDLNMENRPVETYQVHEYLRSKLCSLYENDCIFDKFECCWNGSDSVVMTGSYNNFFRMFDRNTKRDITLEASRENNKPRTVLKPRKVCASGKRKKDEISVDSLDFNKKILHTAWHPKENIIAVATTNNLYIFQDKVN",
# SUMO1_HUMAN
"MSDQEAKPSTEDLGDKKEGEYIKLKVIGQDSSEIHFKVKMTTHLKKLKESYCQRQGVPMNSLRFLFEGQRIADNHTPKELGMEEEDVIEVYQEQTGGHSTV",
# RL38_HUMAN
"MPRKIEEIKDFLLTARRKDAKSVKIKKNKDNVKFKVRCSRYLYTLVITDKEKAEKLKQSLPPGLAVKELK",
# SKP1_HUMAN
"MPSIKLQSSDGEIFEVDVEIAKQSVTIKTMLEDLGMDDEGDDDPVPLPNVNAAILKKVIQWCTHHKDDPPPPEDDENKEKRTDDIPVWDQEFLKVDQGTLFELILAANYLDIKGLLDVTCKTVANMIKGKTPEEIRKTFNIKNDFTEEEEAQVRKENQWCEEK",
# GBG3_HUMAN
"MKGETPVNSTMSIGQARKMVEQLKIEASLCRIKVSKAAADLMTYCDAHACEDPLITPVPTSENPFREKKFFCALL",
# RS21_HUMAN
"MQNDAGEFVDLYVPRKCSASNRIIGAKDHASIQMNVAEVDKVTGRFNGQFKTYAICGAIRRMGESDDSILRLAKADGIVSKNF",
# IF5A1_HUMAN
"MADDLDFETGDAGASATFPMQCSALRKNGFVVLKGRPCKIVEMSTSKTGKHGHAKVHLVGIDIFTGKKYEDICPSTHNMDVPNIKRNDFQLIGIQDGYLSLLQDSGEVREDLRLPEGDLGKEIEQKYDCGEEILITVLSAMTEEAAVAIKAMAK",
# KCNJ2_HUMAN
"MGSVRTNRYSIVSSEEDGMKLATMAVANGFGNGKSKVHTRQQCRSRFVKKDGHCNVQFINVGEKGQRYLADIFTTCVDIRWRWMLVIFCLAFVLSWLFFGCVFWLIALLHGDLDASKEGKACVSEVNSFTAAFLFSIETQTTIGYGFRCVTDECPIAVFMVVFQSIVGCIIDAFIIGAVMAKMAKPKKRNETLVFSHNAVIAMRDGKLCLMWRVGNLRKSHLVEAHVRAQLLKSRITSEGEYIPLDQIDINVGFDSGIDRIFLVSPITIVHEIDEDSPLYDLSKQDIDNADFEIVVILEGMVEATAMTTQCRSSYLANEILWGHRYEPVLFEEKHYYKVDYSRFHKTYEVPNTPLCSARDLAEKKYILSNANSFCYENEVALTSKEEDDSENGVPESTSTDTPPDIDLHNQASVPLEPRPLRRESEI",
# ACTG_HUMAN
"MEEEIAALVIDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVGDEAQSKRGILTLKYPIEHGIVTNWDDMEKIWHHTFYNELRVAPEEHPVLLTEAPLNPKANREKMTQIMFETFNTPAMYVAIQAVLSLYASGRTTGIVMDSGDGVTHTVPIYEGYALPHAILRLDLAGRDLTDYLMKILTERGYSFTTTAEREIVRDIKEKLCYVALDFEQEMATAASSSSLEKSYELPDGQVITIGNERFRCPEALFQPSFLGMESCGIHETTFNSIMKCDVDIRKDLYANTVLSGGTTMYPGIADRMQKEITALAPSTMKIKIIAPPERKYSVWIGGSILASLSTFQQMWISKQEYDESGPSIVHRKCF",
# UBC9_HUMAN
"MSGIALSRLAQERKAWRKDHPFGFVAVPTKNPDGTMNLMNWECAIPGKKGTPWEGGLFKLRMLFKDDYPSSPPKCKFEPPLFHPNVYPSGTVCLSILEEDKDWRPAITIKQILLGIQELLNEPNIQDPAQAEAYTIYCQNRVEYEKRVRAQAKKFAPS",
# PP2AA_HUMAN
"MDEKVFTKELDQWIEQLNECKQLSESQVKSLCEKAKEILTKESNVQEVRCPVTVCGDVHGQFHDLMELFRIGGKSPDTNYLFMGDYVDRGYYSVETVTLLVALKVRYRERITILRGNHESRQITQVYGFYDECLRKYGNANVWKYFTDLFDYLPLTALVDGQIFCLHGGLSPSIDTLDHIRALDRLQEVPHEGPMCDLLWSDPDDRGGWGISPRGAGYTFGQDISETFNHANGLTLVSRAHQLVMEGYNWCHDRNVVTIFSAPNYCYRCGNQAAIMELDDTLKYSFLQFDPAPRRGEPHVTRRTPDYFL",
# YBOX1_HUMAN
"MSSEAETQQPPAAPPAAPALSAADTKPGTTGSGAGSGGPGGLTSAAPAGGDKKVIATKVLGTVKWFNVRNGYGFINRNDTKEDVFVHQTAIKKNNPRKYLRSVGDGETVEFDVVEGEKGAEAANVTGPGGVPVQGSKYAADRNHYRRYPRRRGPPRNYQQNYQNSESGEKNEGSESAPEGQAQQRRPYRRRRFPPYYMRRPYGRRPQYSNPPVQGEVMEGADNQGAGEQGRPVRQNMYRGYRPRFRRGPPRQRQPREDGNEEDKENQGDETQGQQPPQRRYRRNFNYRRRRPENPKPQDGKETKAADPPAENSSAPEAEQGGAE",
# CSK2B_HUMAN
"MSSSEEVSWISWFCGLRGNEFFCEVDEDYIQDKFNLTGLNEQVPHYRQALDMILDLEPDEELEDNPNQSDLIEQAAEMLYGLIHARYILTNRGIAQMLEKYQQGDFGYCPRVYCENQPMLPIGLSDIPGEAMVKLYCPKCMDVYTPKSSRHHHTDGAYFGTGFPHMLFMVHPEYRPKRPANQFVPRLYGFKIHPMAYQLQLQAASNFKSPVKTIR",
# ACTC_HUMAN
"MCDDEETTALVCDNGSGLVKAGFAGDDAPRAVFPSIVGRPRHQGVMVGMGQKDSYVGDEAQSKRGILTLKYPIEHGIITNWDDMEKIWHHTFYNELRVAPEEHPTLLTEAPLNPKANREKMTQIMFETFNVPAMYVAIQAVLSLYASGRTTGIVLDSGDGVTHNVPIYEGYALPHAIMRLDLAGRDLTDYLMKILTERGYSFVTTAEREIVRDIKEKLCYVALDFENEMATAASSSSLEKSYELPDGQVITIGNERFRCPETLFQPSFIGMESAGIHETTYNSIMKCDIDIRKDLYANNVLSGGTTMYPGIADRMQKEITALAPSTMKIKIIAPPERKYSVWIGGSILASLSTFQQMWISKQEYDEAGPSIVHRKCF",
# EF1A1_HUMAN
"MGKEKTHINIVVIGHVDSGKSTTTGHLIYKCGGIDKRTIEKFEKEAAEMGKGSFKYAWVLDKLKAERERGITIDISLWKFETSKYYVTIIDAPGHRDFIKNMITGTSQADCAVLIVAAGVGEFEAGISKNGQTREHALLAYTLGVKQLIVGVNKMDSTEPPYSQKRYEEIVKEVSTYIKKIGYNPDTVAFVPISGWNGDNMLEPSANMPWFKGWKVTRKDGNASGTTLLEALDCILPPTRPTDKPLRLPLQDVYKIGGIGTVPVGRVETGVLKPGMVVTFAPVNVTTEVKSVEMHHEALSEALPGDNVGFNVKNVSVKDVRRGNVAGDSKNDPPMEAAGFTAQVIILNHPGQISAGYAPVLDCHTAHIACKFAELKEKIDRRSGKKLEDGPKFLKSGDAAIVDMVPGKPMCVESFSDYPPLGRFAVRDMRQTVAVGVIKAVDKKAAGAGKVTKSAQKAQKAK",
# CXAR_HUMAN
"MALLLCFVLLCGVVDFARSLSITTPEEMIEKAKGETAYLPCKFTLSPEDQGPLDIEWLISPADNQKVDQVIILYSGDKIYDDYYPDLKGRVHFTSNDLKSGDASINVTNLQLSDIGTYQCKVKKAPGVANKKIHLVVLVKPSGARCYVDGSEEIGSDFKIKCEPKEGSLPLQYEWQKLSDSQKMPTSWLAEMTSSVISVKNASSEYSGTYSCTVRNRVGSDQCLLRLNVVPPSNKAGLIAGAIIGTLLALALIGLIIFCCRKKRREEKYEKEVHHDIREDVPPPKSRTSTARSYIGSNHSSLGSMSPSNMEGYSKTQYNQVPSEDFERTPQSPTLPPAKVAAPNLSRMGAIPVMIPAQSKDGSIV",
# GBRE_HUMAN
"MLSKVLPVLLGILLILQSRVEGPQTESKNEASSRDVVYGPQPQPLENQLLSEETKSTETETGSRVGKLPEASRILNTILSNYDHKLRPGIGEKPTVVTVEISVNSLGPLSILDMEYTIDIIFSQTWYDERLCYNDTFESLVLNGNVVSQLWIPDTFFRNSKRTHEHEITMPNQMVRIYKDGKVLYTIRMTIDAGCSLHMLRFPMDSHSCPLSFSSFSYPENEMIYKWENFKLEINEKNSWKLFQFDFTGVSNKTEIITTPVGDFMVMTIFFNVSRRFGYVAFQNYVPSSVTTMLSWVSFWIKTESAPARTSLGITSVLTMTTLGTFSRKNFPRVSYITALDFYIAICFVFCFCALLEFAVLNFLIYNQTKAHASPKLRHPRINSRAHARTRARSRACARQHQEAFVCQIVTTEGSDGEERPSCSAQQPPSPGSPEGPRSLCSKLACCEWCKRFKKYFCMVPDCEGSTWQQGRLCIHVYRLDNYSRVVFPVTFFFFNVLYWLVCLNL",
# IF4G2_HUMAN
"MESAIAEGGASRFSASSGGGGSRGAPQHYPKTAGNSEFLGKTPGQNAQKWIPARSTRRDDNSAANNSANEKERHDAIFRKVRGILNKLTPEKFDKLCLELLNVGVESKLILKGVILLIVDKALEEPKYSSLYAQLCLRLAEDAPNFDGPAAEGQPGQKQSTTFRRLLISKLQDEFENRTRNVDVYDKRENPLLPEEEEQRAIAKIKMLGNIKFIGELGKLDLIHESILHKCIKTLLEKKKRVQLKDMGEDLECLCQIMRTVGPRLDHERAKSLMDQYFARMCSLMLSKELPARIRFLLQDTVELREHHWVPRKAFLDNGPKTINQIRQDAVKDLGVFIPAPMAQGMRSDFFLEGPFMPPRMKMDRDPLGGLADMFGQMPGSGIGTGPGVIQDRFSPTMGRHRSNQLFNGHGGHIMPPTQSQFGEMGGKFMKSQGLSQLYHNQSQGLLSQLQGQSKDMPPRFSKKGQLNADEISLRPAQSFLMNKNQVPKLQPQITMIPPSAQPPRTQTPPLGQTPQLGLKTNPPLIQEKPAKTSKKPPPSKEELLKLTETVVTEYLNSGNANEAVNGVREMRAPKHFLPEMLSKVIILSLDRSDEDKEKASSLISLLKQEGIATSDNFMQAFLNVLDQCPKLEVDIPLVKSYLAQFAARAIISELVSISELAQPLESGTHFPLFLLCLQQLAKLQDREWLTELFQQSKVNMQKMLPEIDQNKDRMLEILEGKGLSFLFPLLKLEKELLKQIKLDPSPQTIYKWIKDNISPKLHVDKGFVNILMTSFLQYISSEVNPPSDETDSSSAPSKEQLEQEKQLLLSFKPVMQKFLHDHVDLQVSALYALQVHCYNSNFPKGMLLRFFVHFYDMEIIEEEAFLAWKEDITQEFPGKGKALFQVNQWLTWLETAEEEESEEEAD",
# ASIC1_HUMAN
"MELKAEEEEVGGVQPVSIQAFASSSTLHGLAHIFSYERLSLKRALWALCFLGSLAVLLCVCTERVQYYFHYHHVTKLDEVAASQLTFPAVTLCNLNEFRFSQVSKNDLYHAGELLALLNNRYEIPDTQMADEKQLEILQDKANFRSFKPKPFNMREFYDRAGHDIRDMLLSCHFRGEVCSAEDFKVVFTRYGKCYTFNSGRDGRPRLKTMKGGTGNGLEIMLDIQQDEYLPVWGETDETSFEAGIKVQIHSQDEPPFIDQLGFGVAPGFQTFVACQEQRLIYLPPPWGTCKAVTMDSDLDFFDSYSITACRIDCETRYLVENCNCRMVHMPGDAPYCTPEQYKECADPALDFLVEKDQEYCVCEMPCNLTRYGKELSMVKIPSKASAKYLAKKFNKSEQYIGENILVLDIFFEVLNYETIEQKKAYEIAGLLGDIGGQMGLFIGASILTVLELFDYAYEVIKHKLCRRGKCQKEAKRSSADKGVALSLDDVKRHNPCESLRGHPAGMTYAANILPHHPARGTFEDFTC",
# DLG4_HUMAN
"MDCLCIVTTKKYRYQDEDTPPLEHSPAHLPNQANSPPVIVNTDTLEAPGYELQVNGTEGEMEYEEITLERGNSGLGFSIAGGTDNPHIGDDPSIFITKIIPGGAAAQDGRLRVNDSILFVNEVDVREVTHSAAVEALKEAGSIVRLYVMRRKPPAEKVMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGRLQIGDKILAVNSVGLEDVMHEDAVAALKNTYDVVYLKVAKPSNAYLSDSYAPPDITTSYSQHLDNEISHSSYLGTDYPTAMTPTSPRRYSPVAKDLLGEEDIPREPRRIVIHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDQILSVNGVDLRNASHEQAAIALKNAGQTVTIIAQYKPEEYSRFEAKIHDLREQLMNSSLGSGTASLRSNPKRGFYIRALFDYDKTKDCGFLSQALSFRFGDVLHVIDASDEEWWQARRVHSDSETDDIGFIPSKRRVERREWSRLKAKDWGSSSGSQGREDSVLSYETVTQMEVHYARPIIILGPTKDRANDDLLSEFPDKFGSCVPHTTRPKREYEIDGRDYHFVSSREKMEKDIQAHKFIEAGQYNSHLYGTSVQSVREVAEQGKHCILDVSANAVRRLQAAHLHPIAIFIRPRSLENVLEINKRITEEQARKAFDRATKLEQEFTECFSAIVEGDSFEEIYHKVKRVIEDLSGPYIWVPARERL",
# CNTP1_HUMAN
"MMHLRLFCILLAAVSGAEGWGYYGCDEELVGPLYARSLGASSYYSLLTAPRFARLHGISGWSPRIGDPNPWLQIDLMKKHRIRAVATQGSFNSWDWVTRYMLLYGDRVDSWTPFYQRGHNSTFFGNVNESAVVRHDLHFHFTARYIRIVPLAWNPRGKIGLRLGLYGCPYKADILYFDGDDAISYRFPRGVSRSLWDVFAFSFKTEEKDGLLLHAEGAQGDYVTLELEGAHLLLHMSLGSSPIQPRPGHTTVSAGGVLNDQHWHYVRVDRFGRDVNFTLDGYVQRFILNGDFERLNLDTEMFIGGLVGAARKNLAYRHNFRGCIENVIFNRVNIADLAVRRHSRITFEGKVAFRCLDPVPHPINFGGPHNFVQVPGFPRRGRLAVSFRFRTWDLTGLLLFSRLGDGLGHVELTLSEGQVNVSIAQSGRKKLQFAAGYRLNDGFWHEVNFVAQENHAVISIDDVEGAEVRVSYPLLIRTGTSYFFGGCPKPASRWDCHSNQTAFHGCMELLKVDGQLVNLTLVEGRRLGFYAEVLFDTCGITDRCSPNMCEHDGRCYQSWDDFICYCELTGYKGETCHTPLYKESCEAYRLSGKTSGNFTIDPDGSGPLKPFVVYCDIRENRAWTVVRHDRLWTTRVTGSSMERPFLGAIQYWNASWEEVSALANASQHCEQWIEFSCYNSRLLNTAGGYPYSFWIGRNEEQHFYWGGSQPGIQRCACGLDRSCVDPALYCNCDADQPQWRTDKGLLTFVDHLPVTQVVIGDTNRSTSEAQFFLRPLRCYGDRNSWNTISFHTGAALRFPPIRANHSLDVSFYFRTSAPSGVFLENMGGPYCQWRRPYVRVELNTSRDVVFAFDVGNGDENLTVHSDDFEFNDDEWHLVRAEINVKQARLRVDHRPWVLRPMPLQTYIWMEYDQPLYVGSAELKRRPFVGCLRAMRLNGVTLNLEGRANASEGTSPNCTGHCAHPRLPCFHGGRCVERYSYYTCDCDLTAFDGPYCNHDIGGFFEPGTWMRYNLQSALRSAAREFSHMLSRPVPGYEPGYIPGYDTPGYVPGYHGPGYRLPDYPRPGRPVPGYRGPVYNVTGEEVSFSFSTSSAPAVLLYVSSFVRDYMAVLIKDDGTLQLRYQLGTSPYVYQLTTRPVTDGQPHSINITRVYRNLFIQVDYFPLTEQKFSLLVDSQLDSPKALYLGRVMETGVIDPEIQRYNTPGFSGCLSGVRFNNVAPLKTHFRTPRPMTAELAEALRVQGELSESNCGAMPRLVSEVPPELDPWYLPPDFPYYHDEGWVAILLGFLVAFLLLGLVGMLVLFYLQNHRYKGSYHTNEPKAAHEYHPGSKPPLPTSGPAQVPTPTAAPNQAPASAPAPAPTPAPAPGPRDQNLPQILEESRSE",
# SRPK2_HUMAN
"MSVNSEKSSSSERPEPQQKAPLVPPPPPPPPPPPPPLPDPTPPEPEEEILGSDDEEQEDPADYCKGGYHPVKIGDLFNGRYHVIRKLGWGHFSTVWLCWDMQGKRFVAMKVVKSAQHYTETALDEIKLLKCVRESDPSDPNKDMVVQLIDDFKISGMNGIHVCMVFEVLGHHLLKWIIKSNYQGLPVRCVKSIIRQVLQGLDYLHSKCKIIHTDIKPENILMCVDDAYVRRMAAEATEWQKAGAPPPSGSAVSTAPQQKPIGKISKNKKKKLKKKQKRQAELLEKRLQEIEELEREAERKIIEENITSAAPSNDQDGEYCPEVKLKTTGLEEAAEAETAKDNGEAEDQEEKEDAEKENIEKDEDDVDQELANIDPTWIESPKTNGHIENGPFSLEQQLDDEDDDEEDCPNPEEYNLDEPNAESDYTYSSSYEQFNGELPNGRHKIPESQFPEFSTSLFSGSLEPVACGSVLSEGSPLTEQEESSPSHDRSRTVSASSTGDLPKAKTRAADLLVNPLDPRNADKIRVKIADLGNACWVHKHFTEDIQTRQYRSIEVLIGAGYSTPADIWSTACMAFELATGDYLFEPHSGEDYSRDEDHIAHIIELLGSIPRHFALSGKYSREFFNRRGELRHITKLKPWSLFDVLVEKYGWPHEDAAQFTDFLIPMLEMVPEKRASAGECLRHPWLNS",
# RELN_HUMAN
"MERSGWARQTFLLALLLGATLRARAAAGYYPRFSPFFFLCTHHGELEGDGEQGEVLISLHIAGNPTYYVPGQEYHVTISTSTFFDGLLVTGLYTSTSVQASQSIGGSSAFGFGIMSDHQFGNQFMCSVVASHVSHLPTTNLSFIWIAPPAGTGCVNFMATATHRGQVIFKDALAQQLCEQGAPTDVTVHPHLAEIHSDSIILRDDFDSYHQLQLNPNIWVECNNCETGEQCGAIMHGNAVTFCEPYGPRELITTGLNTTTASVLQFSIGSGSCRFSYSDPSIIVLYAKNNSADWIQLEKIRAPSNVSTIIHILYLPEDAKGENVQFQWKQENLRVGEVYEACWALDNILIINSAHRQVVLEDSLDPVDTGNWLFFPGATVKHSCQSDGNSIYFHGNEGSEFNFATTRDVDLSTEDIQEQWSEEFESQPTGWDVLGAVIGTECGTIESGLSMVFLKDGERKLCTPSMDTTGYGNLRFYFVMGGICDPGNSHENDIILYAKIEGRKEHITLDTLSYSSYKVPSLVSVVINPELQTPATKFCLRQKNHQGHNRNVWAVDFFHVLPVLPSTMSHMIQFSINLGCGTHQPGNSVSLEFSTNHGRSWSLLHTECLPEICAGPHLPHSTVYSSENYSGWNRITIPLPNAALTRNTRIRWRQTGPILGNMWAIDNVYIGPSCLKFCSGRGQCTRHGCKCDPGFSGPACEMASQTFPMFISESFGSSRLSSYHNFYSIRGAEVSFGCGVLASGKALVFNKDGRRQLITSFLDSSQSRFLQFTLRLGSKSVLSTCRAPDQPGEGVLLHYSYDNGITWKLLEHYSYLSYHEPRIISVELPGDAKQFGIQFRWWQPYHSSQREDVWAIDEIIMTSVLFNSISLDFTNLVEVTQSLGFYLGNVQPYCGHDWTLCFTGDSKLASSMRYVETQSMQIGASYMIQFSLVMGCGQKYTPHMDNQVKLEYSTNHGLTWHLVQEECLPSMPSCQEFTSASIYHASEFTQWRRVIVLLPQKTWSSATRFRWSQSYYTAQDEWALDSIYIGQQCPNMCSGHGSCDHGICRCDQGYQGTECHPEAALPSTIMSDFENQNGWESDWQEVIGGEIVKPEQGCGVISSGSSLYFSKAGKRQLVSWDLDTSWVDFVQFYIQIGGESASCNKPDSREEGVLLQYSNNGGIQWHLLAEMYFSDFSKPRFVYLELPAAAKTPCTRFRWWQPVFSGEDYDQWAVDDIIILSEKQKQIIPVINPTLPQNFYEKPAFDYPMNQMSVWLMLANEGMVKNETFCAATPSAMIFGKSDGDRFAVTRDLTLKPGYVLQFKLNIGCANQFSSTAPVLLQYSHDAGMSWFLVKEGCYPASAGKGCEGNSRELSEPTMYHTGDFEEWTRITIVIPRSLASSKTRFRWIQESSSQKNVPPFGLDGVYISEPCPSYCSGHGDCISGVCFCDLGYTAAQGTCVSNVPNHNEMFDRFEGKLSPLWYKITGAQVGTGCGTLNDGKSLYFNGPGKREARTVPLDTRNIRLVQFYIQIGSKTSGITCIKPRTRNEGLIVQYSNDNGILWHLLRELDFMSFLEPQIISIDLPQDAKTPATAFRWWQPQHGKHSAQWALDDVLIGMNDSSQTGFQDKFDGSIDLQANWYRIQGGQVDIDCLSMDTALIFTENIGKPRYAETWDFHVSASTFLQFEMSMGCSKPFSNSHSVQLQYSLNNGKDWHLVTEECVPPTIGCLHYTESSIYTSERFQNWKRITVYLPLSTISPRTRFRWIQANYTVGADSWAIDNVVLASGCPWMCSGRGICDAGRCVCDRGFGGPYCVPVVPLPSILKDDFNGNLHPDLWPEVYGAERGNLNGETIKSGTSLIFKGEGLRMLISRDLDCTNTMYVQFSLRFIAKSTPERSHSILLQFSISGGITWHLMDEFYFPQTTNILFINVPLPYTAQTNATRFRLWQPYNNGKKEEIWIVDDFIIDGNNVNNPVMLLDTFDFGPREDNWFFYPGGNIGLYCPYSSKGAPEEDSAMVFVSNEVGEHSITTRDLNVNENTIIQFEINVGCSTDSSSADPVRLEFSRDFGATWHLLLPLCYHSSSHVSSLCSTEHHPSSTYYAGTMQGWRREVVHFGKLHLCGSVRFRWYQGFYPAGSQPVTWAIDNVYIGPQCEEMCNGQGSCINGTKCICDPGYSGPTCKISTKNPDFLKDDFEGQLESDRFLLMSGGKPSRKCGILSSGNNLFFNEDGLRMLMTRDLDLSHARFVQFFMRLGCGKGVPDPRSQPVLLQYSLNGGLSWSLLQEFLFSNSSNVGRYIALEIPLKARSGSTRLRWWQPSENGHFYSPWVIDQILIGGNISGNTVLEDDFTTLDSRKWLLHPGGTKMPVCGSTGDALVFIEKASTRYVVSTDVAVNEDSFLQIDFAASCSVTDSCYAIELEYSVDLGLSWHPLVRDCLPTNVECSRYHLQRILVSDTFNKWTRITLPLPPYTRSQATRFRWHQPAPFDKQQTWAIDNVYIGDGCIDMCSGHGRCIQGNCVCDEQWGGLYCDDPETSLPTQLKDNFNRAPSSQNWLTVNGGKLSTVCGAVASGMALHFSGGCSRLLVTVDLNLTNAEFIQFYFMYGCLITPNNRNQGVLLEYSVNGGITWNLLMEIFYDQYSKPGFVNILLPPDAKEIATRFRWWQPRHDGLDQNDWAIDNVLISGSADQRTVMLDTFSSAPVPQHERSPADAGPVGRIAFDMFMEDKTSVNEHWLFHDDCTVERFCDSPDGVMLCGSHDGREVYAVTHDLTPTEGWIMQFKISVGCKVSEKIAQNQIHVQYSTDFGVSWNYLVPQCLPADPKCSGSVSQPSVFFPTKGWKRITYPLPESLVGNPVRFRFYQKYSDMQWAIDNFYLGPGCLDNCRGHGDCLREQCICDPGYSGPNCYLTHTLKTFLKERFDSEEIKPDLWMSLEGGSTCTECGILAEDTALYFGGSTVRQAVTQDLDLRGAKFLQYWGRIGSENNMTSCHRPICRKEGVLLDYSTDGGITWTLLHEMDYQKYISVRHDYILLPEDALTNTTRLRWWQPFVISNGIVVSGVERAQWALDNILIGGAEINPSQLVDTFDDEGTSHEENWSFYPNAVRTAGFCGNPSFHLYWPNKKKDKTHNALSSRELIIQPGYMMQFKIVVGCEATSCGDLHSVMLEYTKDARSDSWQLVQTQCLPSSSNSIGCSPFQFHEATIYNSVNSSSWKRITIQLPDHVSSSATQFRWIQKGEETEKQSWAIDHVYIGEACPKLCSGHGYCTTGAICICDESFQGDDCSVFSHDLPSYIKDNFESARVTEANWETIQGGVIGSGCGQLAPYAHGDSLYFNGCQIRQAATKPLDLTRASKIMFVLQIGSMSQTDSCNSDLSGPHAVDKAVLLQYSVNNGITWHVIAQHQPKDFTQAQRVSYNVPLEARMKGVLLRWWQPRHNGTGHDQWALDHVEVVLVSTRKQNYMMNFSRQHGLRHFYNRRRRSLRRYP",
# BL1S1_HUMAN
"MAPGSRGERSSFRSRRGPGVPSPQPDVTMLSRLLKEHQAKQNERKELQEKRRREAITAATCLTEALVDHLNVGVAQAYMNQRKLDHEVKTLQVQAAQFAKQTGQWIGMVENFNQALKEIGDVENWARSIELDMRTIATALEYVYKGQLQSAPS",
# MAP1A_HUMAN
"MDGVAEFSEYVSETVDVPSPFDLLEPPTSGGFLKLSKPCCYIFPGGRGDSALFAVNGFNILVDGGSDRKSCFWKLVRHLDRIDSVLLTHIGADNLPGINGLLQRKVAELEEEQSQGSSSYSDWVKNLISPELGVVFFNVPEKLRLPDASRKAKRSIEEACLTLQHLNRLGIQAEPLYRVVSNTIEPLTLFHKMGVGRLDMYVLNPVKDSKEMQFLMQKWAGNSKAKTGIVLPNGKEAEISVPYLTSITALVVWLPANPTEKIVRVLFPGNAPQNKILEGLEKLRHLDFLRYPVATQKDLASGAVPTNLKPSKIKQRADSKESLKATTKTAVSKLAKREEVVEEGAKEARSELAKELAKTEKKAKESSEKPPEKPAKPERVKTESSEALKAEKRKLIKDKVGKKHLKEKISKLEEKKDKEKKEIKKERKELKKDEGRKEEKKDAKKEEKRKDTKPELKKISKPDLKPFTPEVRKTLYKAKVPGRVKIDRSRAIRGEKELSSEPQTPPAQKGTVPLPTISGHRELVLSSPEDLTQDFEEMKREERALLAEQRDTGLGDKPFPLDTAEEGPPSTAIQGTPPSVPGLGQEEHVMKEKELVPEVPEEQGSKDRGLDSGAETEEEKDTWEEKKQREAERLPDRTEAREESEPEVKEDVIEKAELEEMEEVHPSDEEEEDATKAEGFYQKHMQEPLKVTPRSREAFGGRELGLQGKAPEKETSLFLSSLTTPAGATEHVSYIQDETIPGYSETEQTISDEEIHDEPEERPAPPRFHTSTYDLPGPEGAGPFEASQPADSAVPATSGKVYGTPETELTYPTNIVAAPLAEEEHVSSATSITECDKLSSFATSVAEDQSVASLTAPQTEETGKSSLLLDTVTSIPSSRTEATQGLDYVPSAGTISPTSSLEEDKGFKSPPCEDFSVTGESEKRGEIIGKGLSGERAVEEEEEETANVEMSEKLCSQYGTPVFSAPGHALHPGEPALGEAEERCLSPDDSTVKMASPPPSGPPSATHTPFHQSPVEEKSEPQDFQEADSWGDTKRTPGVGKEDAAEETVKPGPEEGTLEKEEKVPPPRSPQAQEAPVNIDEGLTGCTIQLLPAQDKAIVFEIMEAGEPTGPILGAEALPGGLRTLPQEPGKPQKDEVLRYPDRSLSPEDAESLSVLSVPSPDTANQEPTPKSPCGLTEQYLHKDRWPEVSPEDTQSLSLSEESPSKETSLDVSSKQLSPESLGTLQFGELNLGKEEMGHLMQAEDTSHHTAPMSVPEPHAATASPPTDGTTRYSAQTDITDDSLDRKSPASSFSHSTPSGNGKYLPGAITSPDEHILTPDSSFSKSPESLPGPALEDIAIKWEDKVPGLKDRTSEQKKEPEPKDEVLQQKDKTLEHKEVVEPKDTAIYQKDEALHVKNEAVKQQDKALEQKGRDLEQKDTALEQKDKALEPKDKDLEEKDKALEQKDKIPEEKDKALEQKDTALEQKDKALEPKDKDLEQKDRVLEQKEKIPEEKDKALDQKVRSVEHKAPEDTVAEMKDRDLEQTDKAPEQKHQAQEQKDKVSEKKDQALEQKYWALGQKDEALEQNIQALEENHQTQEQESLVQEDKTRKPKMLEEKSPEKVKAMEEKLEALLEKTKALGLEESLVQEGRAREQEEKYWRGQDVVQEWQETSPTREEPAGEQKELAPAWEDTSPEQDNRYWRGREDVALEQDTYWRELSCERKVWFPHELDGQGARPHYTEERESTFLDEGPDDEQEVPLREHATRSPWASDFKDFQESSPQKGLEVERWLAESPVGLPPEEEDKLTRSPFEIISPPASPPEMVGQRVPSAPGQESPIPDPKLMPHMKNEPTTPSWLADIPPWVPKDRPLPPAPLSPAPGPPTPAPESHTPAPFSWGTAEYDSVVAAVQEGAAELEGGPYSPLGKDYRKAEGEREEEGRAEAPDKSSHSSKVPEASKSHATTEPEQTEPEQREPTPYPDERSFQYADIYEQMMLTGLGPACPTREPPLGAAGDWPPCLSTKEAAAGRNTSAEKELSSPISPKSLQSDTPTFSYAALAGPTVPPRPEPGPSMEPSLTPPAVPPRAPILSKGPSPPLNGNILSCSPDRRSPSPKESGRSHWDDSTSDSELEKGAREQPEKEAQSPSPPHPIPMGSPTLWPETEAHVSPPLDSHLGPARPSLDFPASAFGFSSLQPAPPQLPSPAEPRSAPCGSLAFSGDRALALAPGPPTRTRHDEYLEVTKAPSLDSSLPQLPSPSSPGAPLLSNLPRPASPALSEGSSSEATTPVISSVAERFSPSLEAAEQESGELDPGMEPAAHSLWDLTPLSPAPPASLDLALAPAPSLPGDMGDGILPCHLECSEAATEKPSPFQVPSEDCAANGPTETSPNPPGPAPAKAENEEAAACPAWERGAWPEGAERSSRPDTLLSPEQPVCPAGGSGGPPSSASPEVEAGPQGCATEPRPHRGELSPSFLNPPLPPSIDDRDLSTEEVRLVGRGGRRRVGGPGTTGGPCPVTDETPPTSASDSGSSQSDSDVPPETEECPSITAEAALDSDEDGDFLPVDKAGGVSGTHHPRPGHDPPPLPQPDPRPSPPRPDVCMADPEGLSSESGRVERLREKEKVQGRVGRRAPGKAKPASPARRLDLRGKRSPTPGKGPADRASRAPPRPRSTTSQVTPAEEKDGHSPMSKGLVNGLKAGPMALSSKGSSGAPVYVDLAYIPNHCSGKTADLDFFRRVRASYYVVSGNDPANGEPSRAVLDALLEGKAQWGENLQVTLIPTHDTEVTREWYQQTHEQQQQLNVLVLASSSTVVMQDESFPACKIEF",
# GPSM2_HUMAN
"MEENLISMREDHSFHVRYRMEASCLELALEGERLCKSGDCRAGVSFFEAAVQVGTEDLKTLSAIYSQLGNAYFYLHDYAKALEYHHHDLTLARTIGDQLGEAKASGNLGNTLKVLGNFDEAIVCCQRHLDISRELNDKVGEARALYNLGNVYHAKGKSFGCPGPQDVGEFPEEVRDALQAAVDFYEENLSLVTALGDRAAQGRAFGNLGNTHYLLGNFRDAVIAHEQRLLIAKEFGDKAAERRAYSNLGNAYIFLGEFETASEYYKKTLLLARQLKDRAVEAQSCYSLGNTYTLLQDYEKAIDYHLKHLAIAQELNDRIGEGRACWSLGNAYTALGNHDQAMHFAEKHLEISREVGDKSGELTARLNLSDLQMVLGLSYSTNNSIMSENTEIDSSLNGVRPKLGRRHSMENMELMKLTPEKVQNWNSEILAKQKPLIAKPSAKLLFVNRLKGKKYKTNSSTKVLQDASNSIDHRIPNSQRKISADTIGDEGFFDLLSRFQSNRMDDQRCCLQEKNCHTASTTTSSTPPKMMLKTSSVPVVSPNTDEFLDLLASSQSRRLDDQRASFSNLPGLRLTQNSQSVLSHLMTNDNKEADEDFFDILVKCQGSRLDDQRCAPPPATTKGPTVPDEDFFSLILRSQGKRMDEQRVLLQRDQNRDTDFGLKDFLQNNALLEFKNSGKKSADH",
# RL24_HUMAN
"MKVELCSFSGYKIYPGHGRRYARTDGKVFQFLNAKCESAFLSKRNPRQINWTVLYRRKHKKGQSEEIQKKRTRRAVKFQRAITGASLADIMAKRNQKPEVRKAQREQAIRAAKEAKKAKQASKKTAMAAAKAPTKAAPKQKIVKPVKVSAPRVGGKR",
# RL36A_HUMAN
"MVNVPKTRRTFCKKCGKHQPHKVTQYKKGKDSLYAQGKRRYDRKQSGYGGQTKPIFRKKAKTTKKIVLRLECVEPNCRSKRMLAIKRCKHFELGGDKKRKGQVIQF",
# HPCA_HUMAN
"MGKQNSKLRPEMLQDLRENTEFSELELQEWYKGFLKDCPTGILNVDEFKKIYANFFPYGDASKFAEHVFRTFDTNSDGTIDFREFIIALSVTSRGRLEQKLMWAFSMYDLDGNGYISREEMLEIVQAIYKMVSSVMKMPEDESTPEKRTEKIFRQMDTNNDGKLSLEEFIRGAKSDPSIVRLLQCDPSSASQF",
# ARF1_HUMAN
"MGNIFANLFKGLFGKKEMRILMVGLDAAGKTTILYKLKLGEIVTTIPTIGFNVETVEYKNISFTVWDVGGQDKIRPLWRHYFQNTQGLIFVVDSNDRERVNEAREELMRMLAEDELRDAVLLVFANKQDLPNAMNAAEITDKLGLHSLRHRNWYIQATCATSGDGLYEGLDWLSNQLRNQK",
# RHOG_HUMAN
"MQSIKCVVVGDGAVGKTCLLICYTTNAFPKEYIPTVFDNYSAQSAVDGRTVNLNLWDTAGQEEYDRLRTLSYPQTNVFVICFSIASPPSYENVRHKWHPEVCHHCPDVPILLVGTKKDLRAQPDTLRRLKEQGQAPITPQQGQALAKQIHAVRYLECSALQQDGVKEVFAEAVRAVLNPTPIKRGRSCILL",
# RL19_HUMAN
"MSMLRLQKRLASSVLRCGKKKVWLDPNETNEIANANSRQQIRKLIKDGLIIRKPVTVHSRARCRKNTLARRKGRHMGIGKRKGTANARMPEKVTWMRRMRILRRLLRRYRESKKIDRHMYHSLYLKVKGNVFKNKRILMEHIHKLKADKARKKLLADQAEARRSKTKEARKRREERLQAKKEEIIKTLSKEEETKK",
# ALEX_HUMAN
"MMARPVDPQRSPDPTFRSSTRHSGKLEPMEATAHLLRKQCPSRLNSPAWEASGLHWSSLDSPVGSMQALRPSAQHSWSPEPSVVPDQAWEDTALHQKKLCPLSLTSLPREAAVNFSYRSQTLLQEAQVLQGSPELLPRSPKPSGLQRLAPEEATALPLRRLCHLSLMEKDLGTTAHPRGFPELSHKSTAAASSRQSRPRVRSASLPPRTRLPSGSQAPSAAHPKRLSDLLLTSRAAAPGWRSPDPRSRLAAPPLGSTTLPSTWTAPQSRLTARPSRSPEPQIRESEQRDPQLRRKQQRWKEPLMPRREEKYPLRGTDPLPPGQPQRIPLPGQPLQPQPILTPGQPQKIPTPGQHQPILTPGHSQPIPTPGQPLPPQPIPTPGRPLTPQPIPTPGRPLTPQPIQMPGRPLRLPPPLRLLRPGQPMSPQLRQTQGLPLPQPLLPPGQPKSAGRPLQPLPPGPDARSISDPPAPRSRLPIRLLRGLLARLPGGASPRAAAAAACTTMKGWPAATMTPAETSPTMGPPDASAGFSIGEIAAAESPSATYSATFSCKPSGAASVDLRVPSPKPRALSRSRRYPWRRSADRCAKKPWRSGPRSAQRRNAVSSSTNNSRTKRWATCVRTACCF",
# VLDLR_HUMAN
"MGTSALWALWLLLALCWAPRESGATGTGRKAKCEPSQFQCTNGRCITLLWKCDGDEDCVDGSDEKNCVKKTCAESDFVCNNGQCVPSRWKCDGDPDCEDGSDESPEQCHMRTCRIHEISCGAHSTQCIPVSWRCDGENDCDSGEDEENCGNITCSPDEFTCSSGRCISRNFVCNGQDDCSDGSDELDCAPPTCGAHEFQCSTSSCIPISWVCDDDADCSDQSDESLEQCGRQPVIHTKCPASEIQCGSGECIHKKWRCDGDPDCKDGSDEVNCPSRTCRPDQFECEDGSCIHGSRQCNGIRDCVDGSDEVNCKNVNQCLGPGKFKCRSGECIDISKVCNQEQDCRDWSDEPLKECHINECLVNNGGCSHICKDLVIGYECDCAAGFELIDRKTCGDIDECQNPGICSQICINLKGGYKCECSRGYQMDLATGVCKAVGKEPSLIFTNRRDIRKIGLERKEYIQLVEQLRNTVALDADIAAQKLFWADLSQKAIFSASIDDKVGRHVKMIDNVYNPAAIAVDWVYKTIYWTDAASKTISVATLDGTKRKFLFNSDLREPASIAVDPLSGFVYWSDWGEPAKIEKAGMNGFDRRPLVTADIQWPNGITLDLIKSRLYWLDSKLHMLSSVDLNGQDRRIVLKSLEFLAHPLALTIFEDRVYWIDGENEAVYGANKFTGSELATLVNNLNDAQDIIVYHELVQPSGKNWCEEDMENGGCEYLCLPAPQINDHSPKYTCSCPSGYNVEENGRDCQSTATTVTYSETKDTNTTEISATSGLVPGGINVTTAVSEVSVPPKGTSAAWAILPLLLLVMAAVGGYLMWRNWQHKNMKSMNFDNPVYLKTTEEDLSIDIGRHSASVGHTYPAISVVSTDDDLA",
# EFNB1_HUMAN
"MARPGQRWLGKWLVAMVVWALCRLATPLAKNLEPVSWSSLNPKFLSGKGLVIYPKIGDKLDIICPRAEAGRPYEYYKLYLVRPEQAAACSTVLDPNVLVTCNRPEQEIRFTIKFQEFSPNYMGLEFKKHHDYYITSTSNGSLEGLENREGGVCRTRTMKIIMKVGQDPNAVTPEQLTTSRPSKEADNTVKMATQAPGSRGSLGDSDGKHETVNQEEKSGPGASGGSSGDPDGFFNSKVALFAAVGAGCVIFLLIIIFLTVLLLKLRKRHRKHTQQRAAALSLSTLASPKGGSGTAGTEPSDIIIPLRTTENNYCPHYEKVSGDYGHPVYIVQEMPPQSPANIYYKV",
# EM55_HUMAN
"MTLKASEGESGGSMHTALSDLYLEHLLQKRSRPEAVSHPLNTVTEDMYTNGSPAPGSPAQVKGQEVRKVRLIQFEKVTEEPMGITLKLNEKQSCTVARILHGGMIHRQGSLHVGDEILEINGTNVTNHSVDQLQKAMKETKGMISLKVIPNQQSRLPALQMFMRAQFDYDPKKDNLIPCKEAGLKFATGDIIQIINKDDSNWWQGRVEGSSKESAGLIPSPELQEWRVASMAQSAPSEAPSCSPFGKKKKYKDKYLAKHSSIFDQLDVVSYEEVVRLPAFKRKTLVLIGASGVGRSHIKNALLSQNPEKFVYPVPYTTRPPRKSEEDGKEYHFISTEEMTRNISANEFLEFGSYQGNMFGTKFETVHQIHKQNKIAILDIEPQTLKIVRTAELSPFIVFIAPTDQGTQTEALQQLQKDSEAIRSQYAHYFDLSLVNNGVDETLKKLQEAFDQACSSPQWVPVSWVY",
# CDK5_HUMAN
"MQKYEKLEKIGEGTYGTVFKAKNRETHEIVALKRVRLDDDDEGVPSSALREICLLKELKHKNIVRLHDVLHSDKKLTLVFEFCDQDLKKYFDSCNGDLDPEIVKSFLFQLLKGLGFCHSRNVLHRDLKPQNLLINRNGELKLADFGLARAFGIPVRCYSAEVVTLWYRPPDVLFGAKLYSTSIDMWSAGCIFAELANAGRPLFPGNDVDDQLKRIFRLLGTPTEEQWPSMTKLPDYKPYPMYPATTSLVNVVPKLNATGRDLLQNLLKCNPVQRISAEEALQHPYFSDFCPP",
# CDK16_HUMAN
"MDRMKKIKRQLSMTLRGGRGIDKTNGAPEQIGLDESGGGGGSDPGEAPTRAAPGELRSARGPLSSAPEIVHEDLKMGSDGESDQASATSSDEVQSPVRVRMRNHPPRKISTEDINKRLSLPADIRLPEGYLEKLTLNSPIFDKPLSRRLRRVSLSEIGFGKLETYIKLDKLGEGTYATVYKGKSKLTDNLVALKEIRLEHEEGAPCTAIREVSLLKDLKHANIVTLHDIIHTEKSLTLVFEYLDKDLKQYLDDCGNIINMHNVKLFLFQLLRGLAYCHRQKVLHRDLKPQNLLINERGELKLADFGLARAKSIPTKTYSNEVVTLWYRPPDILLGSTDYSTQIDMWGVGCIFYEMATGRPLFPGSTVEEQLHFIFRILGTPTEETWPGILSNEEFKTYNYPKYRAEALLSHAPRLDSDGADLLTKLLQFEGRNRISAEDAMKHPFFLSLGERIHKLPDTTSIFALKEIQLQKEASLRSSSMPDSGRPAFRVVDTEF",
# PURA_HUMAN
"MADRDSGSEQGGAALGSGGSLGHPGSGSGSGGGGGGGGGGGGSGGGGGGAPGGLQHETQELASKRVDIQNKRFYLDVKQNAKGRFLKIAEVGAGGNKSRLTLSMSVAVEFRDYLGDFIEHYAQLGPSQPPDLAQAQDEPRRALKSEFLVRENRKYYMDLKENQRGRFLRIRQTVNRGPGLGSTQGQTIALPAQGLIEFRDALAKLIDDYGVEEEPAELPEGTSLTVDNKRFFFDVGSNKYGVFMRVSEVKPTYRNSITVPYKVWAKFGHTFCKYSEEMKKIQEKQREKRAACEQLHQQQQQQQEETAAATLLLQGEEEGEED",
# CLH1_HUMAN
"MAQILPIRFQEHLQLQNLGINPANIGFSTLTMESDKFICIREKVGEQAQVVIIDMNDPSNPIRRPISADSAIMNPASKVIALKAGKTLQIFNIEMKSKMKAHTMTDDVTFWKWISLNTVALVTDNAVYHWSMEGESQPVKMFDRHSSLAGCQIINYRTDAKQKWLLLTGISAQQNRVVGAMQLYSVDRKVSQPIEGHAASFAQFKMEGNAEESTLFCFAVRGQAGGKLHIIEVGTPPTGNQPFPKKAVDVFFPPEAQNDFPVAMQISEKHDVVFLITKYGYIHLYDLETGTCIYMNRISGETIFVTAPHEATAGIIGVNRKGQVLSVCVEEENIIPYITNVLQNPDLALRMAVRNNLAGAEELFARKFNALFAQGNYSEAAKVAANAPKGILRTPDTIRRFQSVPAQPGQTSPLLQYFGILLDQGQLNKYESLELCRPVLQQGRKQLLEKWLKEDKLECSEELGDLVKSVDPTLALSVYLRANVPNKVIQCFAETGQVQKIVLYAKKVGYTPDWIFLLRNVMRISPDQGQQFAQMLVQDEEPLADITQIVDVFMEYNLIQQCTAFLLDALKNNRPSEGPLQTRLLEMNLMHAPQVADAILGNQMFTHYDRAHIAQLCEKAGLLQRALEHFTDLYDIKRAVVHTHLLNPEWLVNYFGSLSVEDSLECLRAMLSANIRQNLQICVQVASKYHEQLSTQSLIELFESFKSFEGLFYFLGSIVNFSQDPDVHFKYIQAACKTGQIKEVERICRESNCYDPERVKNFLKEAKLTDQLPLIIVCDRFDFVHDLVLYLYRNNLQKYIEIYVQKVNPSRLPVVIGGLLDVDCSEDVIKNLILVVRGQFSTDELVAEVEKRNRLKLLLPWLEARIHEGCEEPATHNALAKIYIDSNNNPERFLRENPYYDSRVVGKYCEKRDPHLACVAYERGQCDLELINVCNENSLFKSLSRYLVRRKDPELWGSVLLESNPYRRPLIDQVVQTALSETQDPEEVSVTVKAFMTADLPNELIELLEKIVLDNSVFSEHRNLQNLLILTAIKADRTRVMEYINRLDNYDAPDIANIAISNELFEEAFAIFRKFDVNTSAVQVLIEHIGNLDRAYEFAERCNEPAVWSQLAKAQLQKGMVKEAIDSYIKADDPSSYMEVVQAANTSGNWEELVKYLQMARKKARESYVETELIFALAKTNRLAELEEFINGPNNAHIQQVGDRCYDEKMYDAAKLLYNNVSNFGRLASTLVHLGEYQAAVDGARKANSTRTWKEVCFACVDGKEFRLAQMCGLHIVVHADELEELINYYQDRGYFEELITMLEAALGLERAHMGMFTELAILYSKFKPQKMREHLELFWSRVNIPKVLRAAEQAHLWAELVFLYDKYEEYDNAIITMMNHPTDAWKEGQFKDIITKVANVELYYRAIQFYLEFKPLLLNDLLMVLSPRLDHTRAVNYFSKVKQLPLVKPYLRSVQNHNNKSVNESLNNLFITEEDYQALRTSIDAYDNFDNISLAQRLEKHELIEFRRIAAYLFKGNNRWKQSVELCKKDSLYKDAMQYASESKDTELAEELLQWFLQEEKRECFGACLFTCYDLLRPDVVLETAWRHNIMDFAMPYFIQVMKEYLTKVDKLDASESLRKEEEQATETQPIVYGQPQLMLTAGPSVAVPPQAPFGYGYTAPPYGQPQPGFGYSM",
# PLCB2_HUMAN
"MSLLNPVLLPPKVKAYLSQGERFIKWDDETTVASPVILRVDPKGYYLYWTYQSKEMEFLDITSIRDTRFGKFAKMPKSQKLRDVFNMDFPDNSFLLKTLTVVSGPDMVDLTFHNFVSYKENVGKAWAEDVLALVKHPLTANASRSTFLDKILVKLKMQLNSEGKIPVKNFFQMFPADRKRVEAALSACHLPKGKNDAINPEDFPEPVYKSFLMSLCPRPEIDEIFTSYHAKAKPYMTKEHLTKFINQKQRDSRLNSLLFPPARPDQVQGLIDKYEPSGINAQRGQLSPEGMVWFLCGPENSVLAQDKLLLHHDMTQPLNHYFINSSHNTYLTAGQFSGLSSAEMYRQVLLSGCRCVELDCWKGKPPDEEPIITHGFTMTTDIFFKEAIEAIAESAFKTSPYPIILSFENHVDSPRQQAKMAEYCRTIFGDMLLTEPLEKFPLKPGVPLPSPEDLRGKILIKNKKNQFSGPTSSSKDTGGEAEGSSPPSAPAGEGTVWAGEEGTELEEEEVEEEEEEESGNLDEEEIKKMQSDEGTAGLEVTAYEEMSSLVNYIQPTKFVSFEFSAQKNRSYVISSFTELKAYDLLSKASVQFVDYNKRQMSRIYPKGTRMDSSNYMPQMFWNAGCQMVALNFQTMDLPMQQNMAVFEFNGQSGYLLKHEFMRRPDKQFNPFSVDRIDVVVATTLSITVISGQFLSERSVRTYVEVELFGLPGDPKRRYRTKLSPSTNSINPVWKEEPFVFEKILMPELASLRVAVMEEGNKFLGHRIIPINALNSGYHHLCLHSESNMPLTMPALFIFLEMKDYIPGAWADLTVALANPIKFFSAHDTKSVKLKEAMGGLPEKPFPLASPVASQVNGALAPTSNGSPAARAGAREEAMKEAAEPRTASLEELRELKGVVKLQRRHEKELRELERRGARRWEELLQRGAAQLAELGPPGVGGVGACKLGPGKGSRKKRSLPREESAGAAPGEGPEGVDGRVRELKDRLELELLRQGEEQYECVLKRKEQHVAEQISKMMELAREKQAAELKALKETSENDTKEMKKKLETKRLERIQGMTKVTTDKMAQERLKREINNSHIQEVVQVIKQMTENLERHQEKLEEKQAACLEQIREMEKQFQKEALAEYEARMKGLEAEVKESVRACLRTCFPSEAKDKPERACECPPELCEQDPLIAKADAQESRL",
# HNRPU_HUMAN
"MSSSPVNVKKLKVSELKEELKKRRLSDKGLKAELMERLQAALDDEEAGGRPAMEPGNGSLDLGGDSAGRSGAGLEQEAAAGGDEEEEEEEEEEEGISALDGDQMELGEENGAAGAADSGPMEEEEAASEDENGDDQGFQEGEDELGDEEEGAGDENGHGEQQPQPPATQQQQPQQQRGAAKEAAGKSSGPTSLFAVTVAPPGARQGQQQAGGKKKAEGGGGGGRPGAPAAGDGKTEQKGGDKKRGVKRPREDHGRGYFEYIEENKYSRAKSPQPPVEEEDEHFDDTVVCLDTYNCDLHFKISRDRLSASSLTMESFAFLWAGGRASYGVSKGKVCFEMKVTEKIPVRHLYTKDIDIHEVRIGWSLTTSGMLLGEEEFSYGYSLKGIKTCNCETEDYGEKFDENDVITCFANFESDEVELSYAKNGQDLGVAFKISKEVLAGRPLFPHVLCHNCAVEFNFGQKEKPYFPIPEEYTFIQNVPLEDRVRGPKGPEEKKDCEVVMMIGLPGAGKTTWVTKHAAENPGKYNILGTNTIMDKMMVAGFKKQMADTGKLNTLLQRAPQCLGKFIEIAARKKRNFILDQTNVSAAAQRRKMCLFAGFQRKAVVVCPKDEDYKQRTQKKAEVEGKDLPEHAVLKMKGNFTLPEVAECFDEITYVELQKEEAQKLLEQYKEESKKALPPEKKQNTGSKKSNKNKSGKNQFNRGGGHRGRGGFNMRGGNFRGGAPGNRGGYNRRGNMPQRGGGGGGSGGIGYPYPRAPVFPGRGSYSNRGNYNRGGMPNRGNYNQNFRGRGNNRGYKNQSQGYNQWQQGQFWGQKPWSQHYHQGYY",
# CAC1B_HUMAN
"MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYKQSIAQRARTMALYNPIPVKQNCFTVNRSLFVFSEDNVVRKYAKRITEWPPFEYMILATIIANCIVLALEQHLPDGDKTPMSERLDDTEPYFIGIFCFEAGIKIIALGFVFHKGSYLRNGWNVMDFVVVLTGILATAGTDFDLRTLRAVRVLRPLKLVSGIPSLQVVLKSIMKAMVPLLQIGLLLFFAILMFAIIGLEFYMGKFHKACFPNSTDAEPVGDFPCGKEAPARLCEGDTECREYWPGPNFGITNFDNILFAILTVFQCITMEGWTDILYNTNDAAGNTWNWLYFIPLIIIGSFFMLNLVLGVLSGEFAKERERVENRRAFLKLRRQQQIERELNGYLEWIFKAEEVMLAEEDRNAEEKSPLDVLKRAATKKSRNDLIHAEEGEDRFADLCAVGSPFARASLKSGKTESSSYFRRKEKMFRFFIRRMVKAQSFYWVVLCVVALNTLCVAMVHYNQPRRLTTTLYFAEFVFLGLFLTEMSLKMYGLGPRSYFRSSFNCFDFGVIVGSVFEVVWAAIKPGSSFGISVLRALRLLRIFKVTKYWSSLRNLVVSLLNSMKSIISLLFLLFLFIVVFALLGMQLFGGQFNFQDETPTTNFDTFPAAILTVFQILTGEDWNAVMYHGIESQGGVSKGMFSSFYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEEEMEEAANQKLALQKAKEVAEVSPMSAANISIAARQQNSAKARSVWEQRASQLRLQNLRASCEALYSEMDPEERLRFATTRHLRPDMKTHLDRPLVVELGRDGARGPVGGKARPEAAEAPEGVDPPRRHHRHRDKDKTPAAGDQDRAEAPKAESGEPGAREERPRPHRSHSKEAAGPPEARSERGRGPGPEGGRRHHRRGSPEEAAEREPRRHRAHRHQDPSKECAGAKGERRARHRGGPRAGPREAESGEEPARRHRARHKAQPAHEAVEKETTEKEATEKEAEIVEADKEKELRNHQPREPHCDLETSGTVTVGPMHTLPSTCLQKVEEQPEDADNQRNVTRMGSQPPDPNTIVHIPVMLTGPLGEATVVPSGNVDLESQAEGKKEVEADDVMRSGPRPIVPYSSMFCLSPTNLLRRFCHYIVTMRYFEVVILVVIALSSIALAAEDPVRTDSPRNNALKYLDYIFTGVFTFEMVIKMIDLGLLLHPGAYFRDLWNILDFIVVSGALVAFAFSGSKGKDINTIKSLRVLRVLRPLKTIKRLPKLKAVFDCVVNSLKNVLNILIVYMLFMFIFAVIAVQLFKGKFFYCTDESKELERDCRGQYLDYEKEEVEAQPRQWKKYDFHYDNVLWALLTLFTVSTGEGWPMVLKHSVDATYEEQGPSPGYRMELSIFYVVYFVVFPFFFVNIFVALIIITFQEQGDKVMSECSLEKNERACIDFAISAKPLTRYMPQNRQSFQYKTWTFVVSPPFEYFIMAMIALNTVVLMMKFYDAPYEYELMLKCLNIVFTSMFSMECVLKIIAFGVLNYFRDAWNVFDFVTVLGSITDILVTEIAETNNFINLSFLRLFRAARLIKLLRQGYTIRILLWTFVQSFKALPYVCLLIAMLFFIYAIIGMQVFGNIALDDDTSINRHNNFRTFLQALMLLFRSATGEAWHEIMLSCLSNQACDEQANATECGSDFAYFYFVSFIFLCSFLMLNLFVAVIMDNFEYLTRDSSILGPHHLDEFIRVWAEYDPAACGRISYNDMFEMLKHMSPPLGLGKKCPARVAYKRLVRMNMPISNEDMTVHFTSTLMALIRTALEIKLAPAGTKQHQCDAELRKEISVVWANLPQKTLDLLVPPHKPDEMTVGKVYAALMIFDFYKQNKTTRDQMQQAPGGLSQMGPVSLFHPLKATLEQTQPAVLRGARVFLRQKSSTSLSNGGAIQNQESGIKESVSWGTQRTQDAPHEARPPLERGHSTEIPVGRSGALAVDVQMQSITRRGPDGEPQPGLESQGRAASMPRLAAETQPVTDASPMKRSISTLAQRPRGTHLCSTTPDRPPPSQASSHHHHHRCHRRRDRKQRSLEKGPSLSADMDGAPSSAVGPGLPPGEGPTGCRRERERRQERGRSQERRQPSSSSSEKQRFYSCDRFGGREPPKPKPSLSSHPTSPTAGQEPGPHPQGSGSVNGSPLLSTSGASTPGRGGRRQLPQTPLTPRPSITYKTANSSPIHFAGAQTSLPAFSPGRLSRGLSEHNALLQRDPLSQPLAPGSRIGSDPYLGQRLDSEASVHALPEDTLTFEEAVATNSGRSSRTSYVSSLTSQSHPLRRVPNGYHCTLGLSSGGRARHSYHHPDQDHWC",
# MDM2_HUMAN
"MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKDTYTMKEVLFYLGQYIMTKRLYDEKQQHIVYCSNDLLGDLFGVPSFSVKEHRKIYTMIYRNLVVVNQQESSDSGTSVSENRCHLEGGSDQKDLVQELQEEKPSSSHLVSRPSTSSRRRAISETEENSDELSGERQRKRHKSDSISLSFDESLALCVIREICCERSSSSESTGTPSNPDLDAGVSEHSGDWLDQDSVSDQFSVEFEVESLDSEDYSLSEEGQELSDEDDEVYQVTVYQAGESDTDSFEEDPEISLADYWKCTSCNEMNPPLPSHCNRCWALRENWLPEDKGKDKGEISEKAKLENSTQAEEGFDVPDCKKTIVNDSRESCVEENDDKITQASQSQESEDYSQPSTSSSIIYSSQEDVKEFEREETQDKEESVESSLPLNAIEPCVICQGRPKNGCIVHGKTGHLMACFTCAKKLKKRNKPCPVCRQPIQMIVLTYFP",
# SPTB2_HUMAN
"MTTTVATDYDNIEIQQQYSDVNNRWDVDDWDNENSSARLFERSRIKALADEREAVQKKTFTKWVNSHLARVSCRITDLYTDLRDGRMLIKLLEVLSGERLPKPTKGRMRIHCLENVDKALQFLKEQRVHLENMGSHDIVDGNHRLTLGLIWTIILRFQIQDISVETEDNKEKKSAKDALLLWCQMKTAGYPNVNIHNFTTSWRDGMAFNALIHKHRPDLIDFDKLKKSNAHYNLQNAFNLAEQHLGLTKLLDPEDISVDHPDEKSIITYVVTYYHYFSKMKALAVEGKRIGKVLDNAIETEKMIEKYESLASDLLEWIEQTIIILNNRKFANSLVGVQQQLQAFNTYRTVEKPPKFTEKGNLEVLLFTIQSKMRANNQKVYMPREGKLISDINKAWERLEKAEHERELALRNELIRQEKLEQLARRFDRKAAMRETWLSENQRLVSQDNFGFDLPAVEAATKKHEAIETDIAAYEERVQAVVAVARELEAENYHDIKRITARKDNVIRLWEYLLELLRARRQRLEMNLGLQKIFQEMLYIMDWMDEMKVLVLSQDYGKHLLGVEDLLQKHTLVEADIGIQAERVRGVNASAQKFATDGEGYKPCDPQVIRDRVAHMEFCYQELCQLAAERRARLEESRRLWKFFWEMAEEEGWIREKEKILSSDDYGKDLTSVMRLLSKHRAFEDEMSGRSGHFEQAIKEGEDMIAEEHFGSEKIRERIIYIREQWANLEQLSAIRKKRLEEASLLHQFQADADDIDAWMLDILKIVSSSDVGHDEYSTQSLVKKHKDVAEEIANYRPTLDTLHEQASALPQEHAESPDVRGRLSGIEERYKEVAELTRLRKQALQDTLALYKMFSEADACELWIDEKEQWLNNMQIPEKLEDLEVIQHRFESLEPEMNNQASRVAVVNQIARQLMHSGHPSEKEIKAQQDKLNTRWSQFRELVDRKKDALLSALSIQNYHLECNETKSWIREKTKVIESTQDLGNDLAGVMALQRKLTGMERDLVAIEAKLSDLQKEAEKLESEHPDQAQAILSRLAEISDVWEEMKTTLKNREASLGEASKLQQFLRDLDDFQSWLSRTQTAIASEDMPNTLTEAEKLLTQHENIKNEIDNYEEDYQKMRDMGEMVTQGQTDAQYMFLRQRLQALDTGWNELHKMWENRQNLLSQSHAYQQFLRDTKQAEAFLNNQEYVLAHTEMPTTLEGAEAAIKKQEDFMTTMDANEEKINAVVETGRRLVSDGNINSDRIQEKVDSIDDRHRKNRETASELLMRLKDNRDLQKFLQDCQELSLWINEKMLTAQDMSYDEARNLHSKWLKHQAFMAELASNKEWLDKIEKEGMQLISEKPETEAVVKEKLTGLHKMWEVLESTTQTKAQRLFDANKAELFTQSCADLDKWLHGLESQIQSDDYGKDLTSVNILLKKQQMLENQMEVRKKEIEELQSQAQALSQEGKSTDEVDSKRLTVQTKFMELLEPLNERKHNLLASKEIHQFNRDVEDEILWVGERMPLATSTDHGHNLQTVQLLIKKNQTLQKEIQGHQPRIDDIFERSQNIVTDSSSLSAEAIRQRLADLKQLWGLLIEETEKRHRRLEEAHRAQQYYFDAAEAEAWMSEQELYMMSEEKAKDEQSAVSMLKKHQILEQAVEDYAETVHQLSKTSRALVADSHPESERISMRQSKVDKLYAGLKDLAEERRGKLDERHRLFQLNREVDDLEQWIAEREVVAGSHELGQDYEHVTMLQERFREFARDTGNIGQERVDTVNHLADELINSGHSDAATIAEWKDGLNEAWADLLELIDTRTQILAASYELHKFYHDAKEIFGRIQDKHKKLPEELGRDQNTVETLQRMHTTFEHDIQALGTQVRQLQEDAARLQAAYAGDKADDIQKRENEVLEAWKSLLDACESRRVRLVDTGDKFRFFSMVRDLMLWMEDVIRQIEAQEKPRDVSSVELLMNNHQGIKAEIDARNDSFTTCIELGKSLLARKHYASEEIKEKLLQLTEKRKEMIDKWEDRWEWLRLILEVHQFSRDASVAEAWLLGQEPYLSSREIGQSVDEVEKLIKRHEAFEKSAATWDERFSALERLTTLELLEVRRQQEEEERKRRPPSPEPSTKVSEEAESQQQWDTSKGEQVSQNGLPAEQGSPRMAETVDTSEMVNGATEQRTSSKESSPIPSPTSDRKAKTALPAQSAATLPARTQETPSAQMEGFLNRKHEWEAHNKKASSRSWHNVYCVINNQEMGFYKDAKTAASGIPYHSEVPVSLKEAVCEVALDYKKKKHVFKLRLNDGNEYLFQAKDDEEMNTWIQAISSAISSDKHEVSASTQSTPASSRAQTLPTSVVTITSESSPGKREKDKEKDKEKRFSLFGKKK",
# RELB_HUMAN
"MLRSGPASGPSVPTGRAMPSRRVARPPAAPELGALGSPDLSSLSLAVSRSTDELEIIDEYIKENGFGLDGGQPGPGEGLPRLVSRGAASLSTVTLGPVAPPATPPPWGCPLGRLVSPAPGPGPQPHLVITEQPKQRGMRFRYECEGRSAGSILGESSTEASKTLPAIELRDCGGLREVEVTACLVWKDWPHRVHPHSLVGKDCTDGICRVRLRPHVSPRHSFNNLGIQCVRKKEIEAAIERKIQLGIDPYNAGSLKNHQEVDMNVVRICFQASYRDQQGQMRRMDPVLSEPVYDKKSTNTSELRICRINKESGPCTGGEELYLLCDKVQKEDISVVFSRASWEGRADFSQADVHRQIAIVFKTPPYEDLEIVEPVTVNVFLQRLTDGVCSEPLPFTYLPRDHDSYGVDKKRKRGMPDVLGELNSSDPHGIESKRRKKKPAILDHFLPNHGSGPFLPPSALLPDPDFFSGTVSLPGLEPPGGPDLLDDGFAYDPTAPTLFTMLDLLPPAPPHASAVVCSGGAGAVVGETPGPEPLTLDSYQAPGPGDGGTASLVGSNMFPNHYREAAFGGGLLSPGPEAT",
# FABP5_HUMAN
"MATVQQLEGRWRLVDSKGFDEYMKELGVGIALRKMGAMAKPDCIITCDGKNLTIKTESTLKTTQFSCTLGEKFEETTADGRKTQTVCNFTDGALVQHQEWDGKESTITRKLKDGKLVVECVMNNVTCTRIYEKVE",
# CAP1_HUMAN
"MADMQNLVERLERAVGRLEAVSHTSDMHRGYADSPSKAGAAPYVQAFDSLLAGPVAEYLKISKEIGGDVQKHAEMVHTGLKLERALLVTASQCQQPAENKLSDLLAPISEQIKEVITFREKNRGSKLFNHLSAVSESIQALGWVAMAPKPGPYVKEMNDAAMFYTNRVLKEYKDVDKKHVDWVKAYLSIWTELQAYIKEFHTTGLAWSKTGPVAKELSGLPSGPSAGSCPPPPPPCPPPPPVSTISCSYESASRSSLFAQINQGESITHALKHVSDDMKTHKNPALKAQSGPVRSGPKPFSAPKPQTSPSPKRATKKEPAVLELEGKKWRVENQENVSNLVIEDTELKQVAYIYKCVNTTLQIKGKINSITVDNCKKLGLVFDDVVGIVEIINSKDVKVQVMGKVPTISINKTDGCHAYLSKNSLDCEIVSAKSSEMNVLIPTEGGDFNEFPVPEQFKTLWNGQKLVTTVTEIAG",
# CAC1D_HUMAN
"MMMMMMMKKMQHQRQQQADHANEANYARGTRLPLSGEGPTSQPNSSKQTVLSWQAAIDAARQAKAAQTMSTSAPPPVGSLSQRKRQQYAKSKKQGNSSNSRPARALFCLSLNNPIRRACISIVEWKPFDIFILLAIFANCVALAIYIPFPEDDSNSTNHNLEKVEYAFLIIFTVETFLKIIAYGLLLHPNAYVRNGWNLLDFVIVIVGLFSVILEQLTKETEGGNHSSGKSGGFDVKALRAFRVLRPLRLVSGVPSLQVVLNSIIKAMVPLLHIALLVLFVIIIYAIIGLELFIGKMHKTCFFADSDIVAEEDPAPCAFSGNGRQCTANGTECRSGWVGPNGGITNFDNFAFAMLTVFQCITMEGWTDVLYWMNDAMGFELPWVYFVSLVIFGSFFVLNLVLGVLSGEFSKEREKAKARGDFQKLREKQQLEEDLKGYLDWITQAEDIDPENEEEGGEEGKRNTSMPTSETESVNTENVSGEGENRGCCGSLCQAISKSKLSRRWRRWNRFNRRRCRAAVKSVTFYWLVIVLVFLNTLTISSEHYNQPDWLTQIQDIANKVLLALFTCEMLVKMYSLGLQAYFVSLFNRFDCFVVCGGITETILVELEIMSPLGISVFRCVRLLRIFKVTRHWTSLSNLVASLLNSMKSIASLLLLLFLFIIIFSLLGMQLFGGKFNFDETQTKRSTFDNFPQALLTVFQILTGEDWNAVMYDGIMAYGGPSSSGMIVCIYFIILFICGNYILLNVFLAIAVDNLADAESLNTAQKEEAEEKERKKIARKESLENKKNNKPEVNQIANSDNKVTIDDYREEDEDKDPYPPCDVPVGEEEEEEEEDEPEVPAGPRPRRISELNMKEKIAPIPEGSAFFILSKTNPIRVGCHKLINHHIFTNLILVFIMLSSAALAAEDPIRSHSFRNTILGYFDYAFTAIFTVEILLKMTTFGAFLHKGAFCRNYFNLLDMLVVGVSLVSFGIQSSAISVVKILRVLRVLRPLRAINRAKGLKHVVQCVFVAIRTIGNIMIVTTLLQFMFACIGVQLFKGKFYRCTDEAKSNPEECRGLFILYKDGDVDSPVVRERIWQNSDFNFDNVLSAMMALFTVSTFEGWPALLYKAIDSNGENIGPIYNHRVEISIFFIIYIIIVAFFMMNIFVGFVIVTFQEQGEKEYKNCELDKNQRQCVEYALKARPLRRYIPKNPYQYKFWYVVNSSPFEYMMFVLIMLNTLCLAMQHYEQSKMFNDAMDILNMVFTGVFTVEMVLKVIAFKPKGYFSDAWNTFDSLIVIGSIIDVALSEADPTESENVPVPTATPGNSEESNRISITFFRLFRVMRLVKLLSRGEGIRTLLWTFIKSFQALPYVALLIAMLFFIYAVIGMQMFGKVAMRDNNQINRNNNFQTFPQAVLLLFRCATGEAWQEIMLACLPGKLCDPESDYNPGEEYTCGSNFAIVYFISFYMLCAFLIINLFVAVIMDNFDYLTRDWSILGPHHLDEFKRIWSEYDPEAKGRIKHLDVVTLLRRIQPPLGFGKLCPHRVACKRLVAMNMPLNSDGTVMFNATLFALVRTALKIKTEGNLEQANEELRAVIKKIWKKTSMKLLDQVVPPAGDDEVTVGKFYATFLIQDYFRKFKKRKEQGLVGKYPAKNTTIALQAGLRTLHDIGPEIRRAISCDLQDDEPEETKREEEDDVFKRNGALLGNHVNHVNSDRRDSLQQTNTTHRPLHVQRPSIPPASDTEKPLFPPAGNSVCHNHHNHNSIGKQVPTSTNANLNNANMSKAAHGKRPSIGNLEHVSENGHHSSHKHDREPQRRSSVKRTRYYETYIRSDSGDEQLPTICREDPEIHGYFRDPHCLGEQEYFSSEECYEDDSSPTWSRQNYGYYSRYPGRNIDSERPRGYHHPQGFLEDDDSPVCYDSRRSPRRRLLPPTPASHRRSSFNFECLRRQSSQEEVPSSPIFPHRTALPLHLMQQQIMAVAGLDSSKAQKYSPSHSTRSWATPPATPPYRDWTPCYTPLIQVEQSEALDQVNGSLPSLHRSSWYTDEPDISYRTFTPASLTVPSSFRNKNSDKQRSADSLVEAVLISEGLGRYARDPKFVSATKHEIADACDLTIDEMESAASTLLNGNVRPRANGDVGPLSHRQDYELQDFGPGYSDEEPDPGRDEEDLADEMICITTL",
# AT2B2_HUMAN
"MGDMTNSDFYSKNQRNESSHGGEFGCTMEELRSLMELRGTEAVVKIKETYGDTEAICRRLKTSPVEGLPGTAPDLEKRKQIFGQNFIPPKKPKTFLQLVWEALQDVTLIILEIAAIISLGLSFYHPPGEGNEGCATAQGGAEDEGEAEAGWIEGAAILLSVICVVLVTAFNDWSKEKQFRGLQSRIEQEQKFTVVRAGQVVQIPVAEIVVGDIAQVKYGDLLPADGLFIQGNDLKIDESSLTGESDQVRKSVDKDPMLLSGTHVMEGSGRMLVTAVGVNSQTGIIFTLLGAGGEEEEKKDKKGVKKGDGLQLPAADGAAASNAADSANASLVNGKMQDGNVDASQSKAKQQDGAAAMEMQPLKSAEGGDADDRKKASMHKKEKSVLQGKLTKLAVQIGKAGLVMSAITVIILVLYFTVDTFVVNKKPWLPECTPVYVQYFVKFFIIGVTVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTTNRMTVVQAYVGDVHYKEIPDPSSINTKTMELLINAIAINSAYTTKILPPEKEGALPRQVGNKTECGLLGFVLDLKQDYEPVRSQMPEEKLYKVYTFNSVRKSMSTVIKLPDESFRMYSKGASEIVLKKCCKILNGAGEPRVFRPRDRDEMVKKVIEPMACDGLRTICVAYRDFPSSPEPDWDNENDILNELTCICVVGIEDPVRPEVPEAIRKCQRAGITVRMVTGDNINTARAIAIKCGIIHPGEDFLCLEGKEFNRRIRNEKGEIEQERIDKIWPKLRVLARSSPTDKHTLVKGIIDSTHTEQRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFSSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTFASLALATEPPTETLLLRKPYGRNKPLISRTMMKNILGHAVYQLALIFTLLFVGEKMFQIDSGRNAPLHSPPSEHYTIIFNTFVMMQLFNEINARKIHGERNVFDGIFRNPIFCTIVLGTFAIQIVIVQFGGKPFSCSPLQLDQWMWCIFIGLGELVWGQVIATIPTSRLKFLKEAGRLTQKEEIPEEELNEDVEEIDHAERELRRGQILWFRGLNRIQTQIRVVKAFRSSLYEGLEKPESRTSIHNFMAHPEFRIEDSQPHIPLIDDTDLEEDAALKQNSSPPSSLNKNNSAIDSGINLTTDTSKSATSSSPGSPIHSLETSL",
# SC6A3_HUMAN
"MSKSKCSVGLMSSVVAPAKEPNAVGPKEVELILVKEQNGVQLTSSTLTNPRQSPVEAQDRETWGKKIDFLLSVIGFAVDLANVWRFPYLCYKNGGGAFLVPYLLFMVIAGMPLFYMELALGQFNREGAAGVWKICPILKGVGFTVILISLYVGFFYNVIIAWALHYLFSSFTTELPWIHCNNSWNSPNCSDAHPGDSSGDSSGLNDTFGTTPAAEYFERGVLHLHQSHGIDDLGPPRWQLTACLVLVIVLLYFSLWKGVKTSGKVVWITATMPYVVLTALLLRGVTLPGAIDGIRAYLSVDFYRLCEASVWIDAATQVCFSLGVGFGVLIAFSSYNKFTNNCYRDAIVTTSINSLTSFSSGFVVFSFLGYMAQKHSVPIGDVAKDGPGLIFIIYPEAIATLPLSSAWAVVFFIMLLTLGIDSAMGGMESVITGLIDEFQLLHRHRELFTLFIVLATFLLSLFCVTNGGIYVFTLLDHFAAGTSILFGVLIEAIGVAWFYGVGQFSDDIQQMTGQRPSLYWRLCWKLVSPCFLLFVVVVSIVTFRPPHYGAYIFPDWANALGWVIATSSMAMVPIYAAYKFCSLPGSFREKLAYAIAPEKDRELVDRGEVRQFTLRHWLKV",
# PLCB3_HUMAN
"MAGAQPGVHALQLEPPTVVETLRRGSKFIKWDEETSSRNLVTLRVDPNGFFLYWTGPNMEVDTLDISSIRDTRTGRYARLPKDPKIREVLGFGGPDARLEEKLMTVVSGPDPVNTVFLNFMAVQDDTAKVWSEELFKLAMNILAQNASRNTFLRKAYTKLKLQVNQDGRIPVKNILKMFSADKKRVETALESCGLKFNRSESIRPDEFSLEIFERFLNKLCLRPDIDKILLEIGAKGKPYLTLEQLMDFINQKQRDPRLNEVLYPPLRPSQARLLIEKYEPNQQFLERDQMSMEGFSRYLGGEENGILPLEALDLSTDMTQPLSAYFINSSHNTYLTAGQLAGTSSVEMYRQALLWGCRCVELDVWKGRPPEEEPFITHGFTMTTEVPLRDVLEAIAETAFKTSPYPVILSFENHVDSAKQQAKMAEYCRSIFGDALLIEPLDKYPLAPGVPLPSPQDLMGRILVKNKKRHRPSAGGPDSAGRKRPLEQSNSALSESSAATEPSSPQLGSPSSDSCPGLSNGEEVGLEKPSLEPQKSLGDEGLNRGPYVLGPADREDEEEDEEEEEQTDPKKPTTDEGTASSEVNATEEMSTLVNYIEPVKFKSFEAARKRNKCFEMSSFVETKAMEQLTKSPMEFVEYNKQQLSRIYPKGTRVDSSNYMPQLFWNVGCQLVALNFQTLDVAMQLNAGVFEYNGRSGYLLKPEFMRRPDKSFDPFTEVIVDGIVANALRVKVISGQFLSDRKVGIYVEVDMFGLPVDTRRKYRTRTSQGNSFNPVWDEEPFDFPKVVLPTLASLRIAAFEEGGKFVGHRILPVSAIRSGYHYVCLRNEANQPLCLPALLIYTEASDYIPDDHQDYAEALINPIKHVSLMDQRARQLAALIGESEAQAGQETCQDTQSQQLGSQPSSNPTPSPLDASPRRPPGPTTSPASTSLSSPGQRDDLIASILSEVAPTPLDELRGHKALVKLRSRQERDLRELRKKHQRKAVTLTRRLLDGLAQAQAEGRCRLRPGALGGAADVEDTKEGEDEAKRYQEFQNRQVQSLLELREAQVDAEAQRRLEHLRQALQRLREVVLDANTTQFKRLKEMNEREKKELQKILDRKRHNSISEAKMRDKHKKEAELTEINRRHITESVNSIRRLEEAQKQRHDRLVAGQQQVLQQLAEEEPKLLAQLAQECQEQRARLPQEIRRSLLGEMPEGLGDGPLVACASNGHAPGSSGHLSGADSESQEENTQL",
# ROR2_HUMAN
"MARGSALPRRPLLCIPAVWAAAALLLSVSRTSGEVEVLDPNDPLGPLDGQDGPIPTLKGYFLNFLEPVNNITIVQGQTAILHCKVAGNPPPNVRWLKNDAPVVQEPRRIIIRKTEYGSRLRIQDLDTTDTGYYQCVATNGMKTITATGVLFVRLGPTHSPNHNFQDDYHEDGFCQPYRGIACARFIGNRTIYVDSLQMQGEIENRITAAFTMIGTSTHLSDQCSQFAIPSFCHFVFPLCDARSRTPKPRELCRDECEVLESDLCRQEYTIARSNPLILMRLQLPKCEALPMPESPDAANCMRIGIPAERLGRYHQCYNGSGMDYRGTASTTKSGHQCQPWALQHPHSHHLSSTDFPELGGGHAYCRNPGGQMEGPWCFTQNKNVRMELCDVPSCSPRDSSKMGILYILVPSIAIPLVIACLFFLVCMCRNKQKASASTPQRRQLMASPSQDMEMPLINQHKQAKLKEISLSAVRFMEELGEDRFGKVYKGHLFGPAPGEQTQAVAIKTLKDKAEGPLREEFRHEAMLRARLQHPNVVCLLGVVTKDQPLSMIFSYCSHGDLHEFLVMRSPHSDVGSTDDDRTVKSALEPPDFVHLVAQIAAGMEYLSSHHVVHKDLATRNVLVYDKLNVKISDLGLFREVYAADYYKLLGNSLLPIRWMAPEAIMYGKFSIDSDIWSYGVVLWEVFSYGLQPYCGYSNQDVVEMIRNRQVLPCPDDCPAWVYALMIECWNEFPSRRPRFKDIHSRLRAWGNLSNYNSSAQTSGASNTTQTSSLSTSPVSNVSNARYVGPKQKAPPFPQPQFIPMKGQIRPMVPPPQLYVPVNGYQPVPAYGAYLPNFYPVQIPMQMAPQQVPPQMVPKPSSHHSGSGSTSTGYVTTAPSNTSMADRAALLSEGADDTQNAPEDGAQSTVQEAEEEEEGSVPETELLGDCDTLQVDEAQVQLEA",
# GCYA1_HUMAN
"MFCTKLKDLKITGECPFSLLAPGQVPNESSEEAAGSSESCKATVPICQDIPEKNIQESLPQRKTSRSRVYLHTLAESICKLIFPEFERLNVALQRTLAKHKIKESRKSLEREDFEKTIAEQAVAAGVPVEVIKESLGEEVFKICYEEDENILGVVGGTLKDFLNSFSTLLKQSSHCQEAGKRGRLEDASILCLDKEDDFLHVYYFFPKRTTSLILPGIIKAAAHVLYETEVEVSLMPPCFHNDCSEFVNQPYLLYSVHMKSTKPSLSPSKPQSSLVIPTSLFCKTFPFHFMFDKDMTILQFGNGIRRLMNRRDFQGKPNFEEYFEILTPKINQTFSGIMTMLNMQFVVRVRRWDNSVKKSSRVMDLKGQMIYIVESSAILFLGSPCVDRLEDFTGRGLYLSDIPIHNALRDVVLIGEQARAQDGLKKRLGKLKATLEQAHQALEEEKKKTVDLLCSIFPCEVAQQLWQGQVVQAKKFSNVTMLFSDIVGFTAICSQCSPLQVITMLNALYTRFDQQCGELDVYKVETIGDAYCVAGGLHKESDTHAVQIALMALKMMELSDEVMSPHGEPIKMRIGLHSGSVFAGVVGVKMPRYCLFGNNVTLANKFESCSVPRKINVSPTTYRLLKDCPGFVFTPRSREELPPNFPSEIPGICHFLDAYQQGTNSKPCFQKKDVEDGNANFLGKASGID",
# GCYB1_HUMAN
"MYGFVNHALELLVIRNYGPEVWEDIKKEAQLDEEGQFLVRIIYDDSKTYDLVAAASKVLNLNAGEILQMFGKMFFVFCQESGYDTILRVLGSNVREFLQNLDALHDHLATIYPGMRAPSFRCTDAEKGKGLILHYYSEREGLQDIVIGIIKTVAQQIHGTEIDMKVIQQRNEECDHTQFLIEEKESKEEDFYEDLDRFEENGTQESRISPYTFCKAFPFHIIFDRDLVVTQCGNAIYRVLPQLQPGNCSLLSVFSLVRPHIDISFHGILSHINTVFVLRSKEGLLDVEKLECEDELTGTEISCLRLKGQMIYLPEADSILFLCSPSVMNLDDLTRRGLYLSDIPLHDATRDLVLLGEQFREEYKLTQELEILTDRLQLTLRALEDEKKKTDTLLYSVLPPSVANELRHKRPVPAKRYDNVTILFSGIVGFNAFCSKHASGEGAMKIVNLLNDLYTRFDTLTDSRKNPFVYKVETVGDKYMTVSGLPEPCIHHARSICHLALDMMEIAGQVQVDGESVQITIGIHTGEVVTGVIGQRMPRYCLFGNTVNLTSRTETTGEKGKINVSEYTYRCLMSPENSDPQFHLEHRGPVSMKGKKEPMQVWFLSRKNTGTEETKQDDD",
# KPCE_HUMAN
"MVVFNGLLKIKICEAVSLKPTAWSLRHAVGPRPQTFLLDPYIALNVDDSRIGQTATKQKTNSPAWHDEFVTDVCNGRKIELAVFHDAPIGYDDFVANCTIQFEELLQNGSRHFEDWIDLEPEGRVYVIIDLSGSSGEAPKDNEERVFRERMRPRKRQGAVRRRVHQVNGHKFMATYLRQPTYCSHCRDFIWGVIGKQGYQCQVCTCVVHKRCHELIITKCAGLKKQETPDQVGSQRFSVNMPHKFGIHNYKVPTFCDHCGSLLWGLLRQGLQCKVCKMNVHRRCETNVAPNCGVDARGIAKVLADLGVTPDKITNSGQRRKKLIAGAESPQPASGSSPSEEDRSKSAPTSPCDQEIKELENNIRKALSFDNRGEEHRAASSPDGQLMSPGENGEVRQGQAKRLGLDEFNFIKVLGKGSFGKVMLAELKGKDEVYAVKVLKKDVILQDDDVDCTMTEKRILALARKHPYLTQLYCCFQTKDRLFFVMEYVNGGDLMFQIQRSRKFDEPRSRFYAAEVTSALMFLHQHGVIYRDLKLDNILLDAEGHCKLADFGMCKEGILNGVTTTTFCGTPDYIAPEILQELEYGPSVDWWALGVLMYEMMAGQPPFEADNEDDLFESILHDDVLYPVWLSKEAVSILKAFMTKNPHKRLGCVASQNGEDAIKQHPFFKEIDWVLLEQKKIKPPFKPRIKTKRDVNNFDQDFTREEPVLTLVDEAIVKQINQEEFKGFSYFGEDLMP",
# CNTN2_HUMAN
"MGTATRRKPHLLLVAAVALVSSSAWSSALGSQTTFGPVFEDQPLSVLFPEESTEEQVLLACRARASPPATYRWKMNGTEMKLEPGSRHQLVGGNLVIMNPTKAQDAGVYQCLASNPVGTVVSREAILRFGFLQEFSKEERDPVKAHEGWGVMLPCNPPAHYPGLSYRWLLNEFPNFIPTDGRHFVSQTTGNLYIARTNASDLGNYSCLATSHMDFSTKSVFSKFAQLNLAAEDTRLFAPSIKARFPAETYALVGQQVTLECFAFGNPVPRIKWRKVDGSLSPQWTTAEPTLQIPSVSFEDEGTYECEAENSKGRDTVQGRIIVQAQPEWLKVISDTEADIGSNLRWGCAAAGKPRPTVRWLRNGEPLASQNRVEVLAGDLRFSKLSLEDSGMYQCVAENKHGTIYASAELAVQALAPDFRLNPVRRLIPAARGGEILIPCQPRAAPKAVVLWSKGTEILVNSSRVTVTPDGTLIIRNISRSDEGKYTCFAENFMGKANSTGILSVRDATKITLAPSSADINLGDNLTLQCHASHDPTMDLTFTWTLDDFPIDFDKPGGHYRRTNVKETIGDLTILNAQLRHGGKYTCMAQTVVDSASKEATVLVRGPPGPPGGVVVRDIGDTTIQLSWSRGFDNHSPIAKYTLQARTPPAGKWKQVRTNPANIEGNAETAQVLGLTPWMDYEFRVIASNILGTGEPSGPSSKIRTREAAPSVAPSGLSGGGGAPGELIVNWTPMSREYQNGDGFGYLLSFRRQGSTHWQTARVPGADAQYFVYSNESVRPYTPFEVKIRSYNRRGDGPESLTALVYSAEEEPRVAPTKVWAKGVSSSEMNVTWEPVQQDMNGILLGYEIRYWKAGDKEAAADRVRTAGLDTSARVSGLHPNTKYHVTVRAYNRAGTGPASPSANATTMKPPPRRPPGNISWTFSSSSLSIKWDPVVPFRNESAVTGYKMLYQNDLHLTPTLHLTGKNWIEIPVPEDIGHALVQIRTTGPGGDGIPAEVHIVRNGGTSMMVENMAVRPAPHPGTVISHSVAMLILIGSLEL",
# NRG1_HUMAN
"MSERKEGRGKGKGKKKERGSGKKPESAAGSQSPALPPRLKEMKSQESAAGSKLVLRCETSSEYSSLRFKWFKNGNELNRKNKPQNIKIQKKPGKSELRINKASLADSGEYMCKVISKLGNDSASANITIVESNEIITGMPASTEGAYVSSESPIRISVSTEGANTSSSTSTSTTGTSHLVKCAEKEKTFCVNGGECFMVKDLSNPSRYLCKCQPGFTGARCTENVPMKVQNQEKAEELYQKRVLTITGICIALLVVGIMCVVAYCKTKKQRKKLHDRLRQSLRSERNNMMNIANGPHHPNPPPENVQLVNQYVSKNVISSEHIVEREAETSFSTSHYTSTAHHSTTVTQTPSHSWSNGHTESILSESHSVIVMSSVENSRHSSPTGGPRGRLNGTGGPRECNSFLRHARETPDSYRDSPHSERYVSAMTTPARMSPVDFHTPSSPKSPPSEMSPPVSSMTVSMPSMAVSPFMEEERPLLLVTPPRLREKKFDHHPQQFSSFHHNPAHDSNSLPASPLRIVEDEEYETTQEYEPAQEPVKKLANSRRAKRTKPNGHIANRLEVDSNTSSQSSNSESETEDERVGEDTPFLGIQNPLAASLEATPAFRLADSRTNPAGRFSTQEEIQARLSSVIANQDPIAV",
# APBA1_HUMAN
"MNHLEGSAEVEVTDEAAGGEVNESVEADLEHPEVEEEQQQPPQQQHYVGRHQRGRALEDLRAQLGQEEEERGECLARSASTESGFHNHTDTAEGDVIAAARDGYDAERAQDPEDESAYAVQYRPEAEEYTEQAEAEHAEATHRRALPNHLHFHSLEHEEAMNAAYSGYVYTHRLFHRGEDEPYSEPYADYGGLQEHVYEEIGDAPELDARDGLRLYEQERDEAAAYRQEALGARLHHYDERSDGESDSPEKEAEFAPYPRMDSYEQEEDIDQIVAEVKQSMSSQSLDKAAEDMPEAEQDLERPPTPAGGRPDSPGLQAPAGQQRAVGPAGGGEAGQRYSKEKRDAISLAIKDIKEAIEEVKTRTIRSPYTPDEPKEPIWVMRQDISPTRDCDDQRPMDGDSPSPGSSSPLGAESSSTSLHPSDPVEASTNKESRKSLASFPTYVEVPGPCDPEDLIDGIIFAANYLGSTQLLSDKTPSKNVRMMQAQEAVSRIKMAQKLAKSRKKAPEGESQPMTEVDLFISTQRIKVLNADTQETMMDHPLRTISYIADIGNIVVLMARRRMPRSNSQENVEASHPSQDGKRQYKMICHVFESEDAQLIAQSIGQAFSVAYQEFLRANGINPEDLSQKEYSDLLNTQDMYNDDLIHFSKSENCKDVFIEKQKGEILGVVIVESGWGSILPTVIIANMMHGGPAEKSGKLNIGDQIMSINGTSLVGLPLSTCQSIIKGLKNQSRVKLNIVRCPPVTTVLIRRPDLRYQLGFSVQNGIICSLMRGGIAERGGVRVGHRIIEINGQSVVATPHEKIVHILSNAVGEIHMKTMPAAMYRLLTAQEQPVYI",
# RL18A_HUMAN
"MKASGTLREYKVVGRCLPTPKCHTPPLYRMRIFAPNHVVAKSRFWYFVSQLKKMKKSSGEIVYCGQVFEKSPLRVKNFGIWLRYDSRSGTHNMYREYRDLTTAGAVTQCYRDMGARHRARAHSIQIMKVEEIAASKCRRPAVKQFHDSKIKFPLPHRVLRRQHKPRFTTKRPNTFF",
# CACB1_HUMAN
"MVQKTSMSRGPYPPSQEIPMEVFDPSPQGKYSKRKGRFKRSDGSTSSDTTSNSFVRQGSAESYTSRPSDSDVSLEEDREALRKEAERQALAQLEKAKTKPVAFAVRTNVGYNPSPGDEVPVQGVAITFEPKDFLHIKEKYNNDWWIGRLVKEGCEVGFIPSPVKLDSLRLLQEQKLRQNRLGSSKSGDNSSSSLGDVVTGTRRPTPPASAKQKQKSTEHVPPYDVVPSMRPIILVGPSLKGYEVTDMMQKALFDFLKHRFDGRISITRVTADISLAKRSVLNNPSKHIIIERSNTRSSLAEVQSEIERIFELARTLQLVALDADTINHPAQLSKTSLAPIIVYIKITSPKVLQRLIKSRGKSQSKHLNVQIAASEKLAQCPPEMFDIILDENQLEDACEHLAEYLEAYWKATHPPSSTPPNPLLNRTMATAALAASPAPVSNLQGPYLASGDQPLERATGEHASMHEYPGELGQPPGLYPSSHPPGRAGTLRALSRQDTFDADTPGSRNSAYTELGDSCVDMETDPSEGPGLGDPAGGGTPPARQGSWEDEEEDYEEELTDNRNRGRNKARYCAEGGGPVLGRNKNELEGWGRGVYIR",
# MP2K1_HUMAN
"MPKKKPTPIQLNPAPDGSAVNGTSSAETNLEALQKKLEELELDEQQRKRLEAFLTQKQKVGELKDDDFEKISELGAGNGGVVFKVSHKPSGLVMARKLIHLEIKPAIRNQIIRELQVLHECNSPYIVGFYGAFYSDGEISICMEHMDGGSLDQVLKKAGRIPEQILGKVSIAVIKGLTYLREKHKIMHRDVKPSNILVNSRGEIKLCDFGVSGQLIDSMANSFVGTRSYMSPERLQGTHYSVQSDIWSMGLSLVEMAVGRYPIPPPDAKELELMFGCQVEGDAAETPPRPRTPGRPLSSYGMDSRPPMAIFELLDYIVNEPPPKLPSGVFSLEFQDFVNKCLIKNPAERADLKQLMVHAFIKRSDAEEVDFAGWLCSTIGLNQPSTPTHAAGV",
# RL6_HUMAN
"MAGEKVEKPDTKEKKPEAKKVDAGGKVKKGNLKAKKPKKGKPHCSRNPVLVRGIGRYSRSAMYSRKAMYKRKYSAAKSKVEKKKKEKVLATVTKPVGGDKNGGTRVVKLRKMPRYYPTEDVPRKLLSHGKKPFSQHVRKLRASITPGTILIILTGRHRGKRVVFLKQLASGLLLVTGPLVLNRVPLRRTHQKFVIATSTKIDISNVKIPKHLTDAYFKKKKLRKPRHQEGEIFDTEKEKYEITEQRKIDQKAVDSQILPKIKAIPQLQGYLRSVFALTNGIYPHKLVF",
# AKA12_HUMAN
"MGAGSSTEQRSPEQPPEGSSTPAEPEPSGGGPSAEAAPDTTADPAIAASDPATKLLQKNGQLSTINGVAEQDELSLQEGDLNGQKGALNGQGALNSQEEEEVIVTEVGQRDSEDVSKRDSDKEMATKSAVVHDITDDGQEETPEIIEQIPSSESNLEELTQPTESQANDIGFKKVFKFVGFKFTVKKDKTEKPDTVQLLTVKKDEGEGAAGAGDHKDPSLGAGEAASKESEPKQSTEKPEETLKREQSHAEISPPAESGQAVEECKEEGEEKQEKEPSKSAESPTSPVTSETGSTFKKFFTQGWAGWRKKTSFRKPKEDEVEASEKKKEQEPEKVDTEEDGKAEVASEKLTASEQAHPQEPAESAHEPRLSAEYEKVELPSEEQVSGSQGPSEEKPAPLATEVFDEKIEVHQEEVVAEVHVSTVEERTEEQKTEVEETAGSVPAEELVEMDAEPQEAEPAKELVKLKETCVSGEDPTQGADLSPDEKVLSKPPEGVVSEVEMLSSQERMKVQGSPLKKLFTSTGLKKLSGKKQKGKRGGGDEESGEHTQVPADSPDSQEEQKGESSASSPEEPEEITCLEKGLAEVQQDGEAEEGATSDGEKKREGVTPWASFKKMVTPKKRVRRPSESDKEDELDKVKSATLSSTESTASEMQEEMKGSVEEPKPEEPKRKVDTSVSWEALICVGSSKKRARRGSSSDEEGGPKAMGGDHQKADEAGKDKETGTDGILAGSQEHDPGQGSSSPEQAGSPTEGEGVSTWESFKRLVTPRKKSKSKLEEKSEDSIAGSGVEHSTPDTEPGKEESWVSIKKFIPGRRKKRPDGKQEQAPVEDAGPTGANEDDSDVPAVVPLSEYDAVEREKMEAQQAQKSAEQPEQKAATEVSKELSESQVHMMAAAVADGTRAATIIEERSPSWISASVTEPLEQVEAEAALLTEEVLEREVIAEEEPPTVTEPLPENREARGDTVVSEAELTPEAVTAAETAGPLGAEEGTEASAAEETTEMVSAVSQLTDSPDTTEEATPVQEVEGGVPDIEEQERRTQEVLQAVAEKVKEESQLPGTGGPEDVLQPVQRAEAERPEEQAEASGLKKETDVVLKVDAQEAKTEPFTQGKVVGQTTPESFEKAPQVTESIESSELVTTCQAETLAGVKSQEMVMEQAIPPDSVETPTDSETDGSTPVADFDAPGTTQKDEIVEIHEENEVASGTQSGGTEAEAVPAQKERPPAPSSFVFQEETKEQSKMEDTLEHTDKEVSVETVSILSKTEGTQEADQYADEKTKDVPFFEGLEGSIDTGITVSREKVTEVALKGEGTEEAECKKDDALELQSHAKSPPSPVEREMVVQVEREKTEAEPTHVNEEKLEHETAVTVSEEVSKQLLQTVNVPIIDGAKEVSSLEGSPPPCLGQEEAVCTKIQVQSSEASFTLTAAAEEEKVLGETANILETGETLEPAGAHLVLEEKSSEKNEDFAAHPGEDAVPTGPDCQAKSTPVIVSATTKKGLSSDLEGEKTTSLKWKSDEVDEQVACQEVKVSVAIEDLEPENGILELETKSSKLVQNIIQTAVDQFVRTEETATEMLTSELQTQAHVIKADSQDAGQETEKEGEEPQASAQDETPITSAKEESESTAVGQAHSDISKDMSEASEKTMTVEVEGSTVNDQQLEEVVLPSEEEGGGAGTKSVPEDDGHALLAERIEKSLVEPKEDEKGDDVDDPENQNSALADTDASGGLTKESPDTNGPKQKEKEDAQEVELQEGKVHSESDKAITPQAQEELQKQERESAKSELTES",
# KCNC4_HUMAN
"MISSVCVSSYRGRKSGNKPPSKTCLKEEMAKGEASEKIIINVGGTRHETYRSTLRTLPGTRLAWLADPDGGGRPETDGGGVGSSGSSGGGGCEFFFDRHPGVFAYVLNYYRTGKLHCPADVCGPLFEEELTFWGIDETDVEPCCWMTYRQHRDAEEALDIFESPDGGGSGAGPSDEAGDDERELALQRLGPHEGGAGHGAGSGGCRGWQPRMWALFEDPYSSRAARVVAFASLFFILVSITTFCLETHEAFNIDRNVTEILRVGNITSVHFRREVETEPILTYIEGVCVLWFTLEFLVRIVCCPDTLDFVKNLLNIIDFVAILPFYLEVGLSGLSSKAARDVLGFLRVVRFVRILRIFKLTRHFVGLRVLGHTLRASTNEFLLLIIFLALGVLIFATMIYYAERIGARPSDPRGNDHTDFKNIPIGFWWAVVTMTTLGYGDMYPKTWSGMLVGALCALAGVLTIAMPVPVIVNNFGMYYSLAMAKQKLPKKRKKHVPRPAQLESPMYCKSEETSPRDSTCSDTSPPAREEGMIERKRADSKQNGDANAVLSDEEGAGLTQPLASSPTPEERRALRRSTTRDRNKKAAACFLLSTGDYACADGSVRKGTFVLRDLPLQHSPEAACPPTAGTLFLPH",
# TF65_HUMAN
"MDELFPLIFPAEPAQASGPYVEIIEQPKQRGMRFRYKCEGRSAGSIPGERSTDTTKTHPTIKINGYTGPGTVRISLVTKDPPHRPHPHELVGKDCRDGFYEAELCPDRCIHSFQNLGIQCVKKRDLEQAISQRIQTNNNPFQVPIEEQRGDYDLNAVRLCFQVTVRDPSGRPLRLPPVLSHPIFDNRAPNTAELKICRVNRNSGSCLGGDEIFLLCDKVQKEDIEVYFTGPGWEARGSFSQADVHRQVAIVFRTPPYADPSLQAPVRVSMQLRRPSDRELSEPMEFQYLPDTDDRHRIEEKRKRTYETFKSIMKKSPFSGPTDPRPPPRRIAVPSRSSASVPKPAPQPYPFTSSLSTINYDEFPTMVFPSGQISQASALAPAPPQVLPQAPAPAPAPAMVSALAQAPAPVPVLAPGPPQAVAPPAPKPTQAGEGTLSEALLQLQFDDEDLGALLGNSTDPAVFTDLASVDNSEFQQLLNQGIPVAPHTTEPMLMEYPEAITRLVTGAQRPPDPAPAPLGAPGLPNGLLSGDEDFSSIADMDFSALLSQISS",
# IF4G1_HUMAN
"MNKAPQSTGPPPAPSPGLPQPAFPPGQTAPVVFSTPQATQMNTPSQPRQHFYPSRAQPPSSAASRVQSAAPARPGPAAHVYPAGSQVMMIPSQISYPASQGAYYIPGQGRSTYVVPTQQYPVQPGAPGFYPGASPTEFGTYAGAYYPAQGVQQFPTGVAPTPVLMNQPPQIAPKRERKTIRIRDPNQGGKDITEEIMSGARTASTPTPPQTGGGLEPQANGETPQVAVIVRPDDRSQGAIIADRPGLPGPEHSPSESQPSSPSPTPSPSPVLEPGSEPNLAVLSIPGDTMTTIQMSVEESTPISRETGEPYRLSPEPTPLAEPILEVEVTLSKPVPESEFSSSPLQAPTPLASHTVEIHEPNGMVPSEDLEPEVESSPELAPPPACPSESPVPIAPTAQPEELLNGAPSPPAVDLSPVSEPEEQAKEVTASMAPPTIPSATPATAPSATSPAQEEEMEEEEEEEEGEAGEAGEAESEKGGEELLPPESTPIPANLSQNLEAAAATQVAVSVPKRRRKIKELNKKEAVGDLLDAFKEANPAVPEVENQPPAGSNPGPESEGSGVPPRPEEADETWDSKEDKIHNAENIQPGEQKYEYKSDQWKPLNLEEKKRYDREFLLGFQFIFASMQKPEGLPHISDVVLDKANKTPLRPLDPTRLQGINCGPDFTPSFANLGRTTLSTRGPPRGGPGGELPRGPAGLGPRRSQQGPRKEPRKIIATVLMTEDIKLNKAEKAWKPSSKRTAADKDRGEEDADGSKTQDLFRRVRSILNKLTPQMFQQLMKQVTQLAIDTEERLKGVIDLIFEKAISEPNFSVAYANMCRCLMALKVPTTEKPTVTVNFRKLLLNRCQKEFEKDKDDDEVFEKKQKEMDEAATAEERGRLKEELEEARDIARRRSLGNIKFIGELFKLKMLTEAIMHDCVVKLLKNHDEESLECLCRLLTTIGKDLDFEKAKPRMDQYFNQMEKIIKEKKTSSRIRFMLQDVLDLRGSNWVPRRGDQGPKTIDQIHKEAEMEEHREHIKVQQLMAKGSDKRRGGPPGPPISRGLPLVDDGGWNTVPISKGSRPIDTSRLTKITKPGSIDSNNQLFAPGGRLSWGKGSSGGSGAKPSDAASEAARPATSTLNRFSALQQAVPTESTDNRRVVQRSSLSRERGEKAGDRGDRLERSERGGDRGDRLDRARTPATKRSFSKEVEERSRERPSQPEGLRKAASLTEDRDRGRDAVKREAALPPVSPLKAALSEEELEKKSKAIIEEYLHLNDMKEAVQCVQELASPSLLFIFVRHGVESTLERSAIAREHMGQLLHQLLCAGHLSTAQYYQGLYEILELAEDMEIDIPHVWLYLAELVTPILQEGGVPMGELFREITKPLRPLGKAASLLLEILGLLCKSMGPKKVGTLWREAGLSWKEFLPEGQDIGAFVAEQKVEYTLGEESEAPGQRALPSEELNRQLEKLLKEGSSNQRVFDWIEANLSEQQIVSNTLVRALMTAVCYSAIIFETPLRVDVAVLKARAKLLQKYLCDEQKELQALYALQALVVTLEQPPNLLRMFFDALYDEDVVKEDAFYSWESSKDPAEQQGKGVALKSVTAFFKWLREAEEESDHN",
# NOTC2_HUMAN
"MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHNGTGYCKCPEGFLGEYCQHRDPCEKNRCQNGGTCVAQAMLGKATCRCASGFTGEDCQYSTSHPCFVSRPCLNGGTCHMLSRDTYECTCQVGFTGKECQWTDACLSHPCANGSTCTTVANQFSCKCLTGFTGQKCETDVNECDIPGHCQHGGTCLNLPGSYQCQCPQGFTGQYCDSLYVPCAPSPCVNGGTCRQTGDFTFECNCLPGFEGSTCERNIDDCPNHRCQNGGVCVDGVNTYNCRCPPQWTGQFCTEDVDECLLQPNACQNGGTCANRNGGYGCVCVNGWSGDDCSENIDDCAFASCTPGSTCIDRVASFSCMCPEGKAGLLCHLDDACISNPCHKGALCDTNPLNGQYICTCPQGYKGADCTEDVDECAMANSNPCEHAGKCVNTDGAFHCECLKGYAGPRCEMDINECHSDPCQNDATCLDKIGGFTCLCMPGFKGVHCELEINECQSNPCVNNGQCVDKVNRFQCLCPPGFTGPVCQIDIDDCSSTPCLNGAKCIDHPNGYECQCATGFTGVLCEENIDNCDPDPCHHGQCQDGIDSYTCICNPGYMGAICSDQIDECYSSPCLNDGRCIDLVNGYQCNCQPGTSGVNCEINFDDCASNPCIHGICMDGINRYSCVCSPGFTGQRCNIDIDECASNPCRKGATCINGVNGFRCICPEGPHHPSCYSQVNECLSNPCIHGNCTGGLSGYKCLCDAGWVGINCEVDKNECLSNPCQNGGTCDNLVNGYRCTCKKGFKGYNCQVNIDECASNPCLNQGTCFDDISGYTCHCVLPYTGKNCQTVLAPCSPNPCENAAVCKESPNFESYTCLCAPGWQGQRCTIDIDECISKPCMNHGLCHNTQGSYMCECPPGFSGMDCEEDIDDCLANPCQNGGSCMDGVNTFSCLCLPGFTGDKCQTDMNECLSEPCKNGGTCSDYVNSYTCKCQAGFDGVHCENNINECTESSCFNGGTCVDGINSFSCLCPVGFTGSFCLHEINECSSHPCLNEGTCVDGLGTYRCSCPLGYTGKNCQTLVNLCSRSPCKNKGTCVQKKAESQCLCPSGWAGAYCDVPNVSCDIAASRRGVLVEHLCQHSGVCINAGNTHYCQCPLGYTGSYCEEQLDECASNPCQHGATCSDFIGGYRCECVPGYQGVNCEYEVDECQNQPCQNGGTCIDLVNHFKCSCPPGTRGLLCEENIDDCARGPHCLNGGQCMDRIGGYSCRCLPGFAGERCEGDINECLSNPCSSEGSLDCIQLTNDYLCVCRSAFTGRHCETFVDVCPQMPCLNGGTCAVASNMPDGFICRCPPGFSGARCQSSCGQVKCRKGEQCVHTASGPRCFCPSPRDCESGCASSPCQHGGSCHPQRQPPYYSCQCAPPFSGSRCELYTAPPSTPPATCLSQYCADKARDGVCDEACNSHACQWDGGDCSLTMENPWANCSSPLPCWDYINNQCDELCNTVECLFDNFECQGNSKTCKYDKYCADHFKDNHCDQGCNSEECGWDGLDCAADQPENLAEGTLVIVVLMPPEQLLQDARSFLRALGTLLHTNLRIKRDSQGELMVYPYYGEKSAAMKKQRMTRRSLPGEQEQEVAGSKVFLEIDNRQCVQDSDHCFKNTDAAAALLASHAIQGTLSYPLVSVVSESLTPERTQLLYLLAVAVVIILFIILLGVIMAKRKRKHGSLWLPEGFTLRRDASNHKRREPVGQDAVGLKNLSVQVSEANLIGTGTSEHWVDDEGPQPKKVKAEDEALLSEEDDPIDRRPWTQQHLEAADIRRTPSLALTPPQAEQEVDVLDVNVRGPDGCTPLMLASLRGGSSDLSDEDEDAEDSSANIITDLVYQGASLQAQTDRTGEMALHLAARYSRADAAKRLLDAGADANAQDNMGRCPLHAAVAADAQGVFQILIRNRVTDLDARMNDGTTPLILAARLAVEGMVAELINCQADVNAVDDHGKSALHWAAAVNNVEATLLLLKNGANRDMQDNKEETPLFLAAREGSYEAAKILLDHFANRDITDHMDRLPRDVARDRMHHDIVRLLDEYNVTPSPPGTVLTSALSPVICGPNRSFLSLKHTPMGKKSRRPSAKSTMPTSLPNLAKEAKDAKGSRRKKSLSEKVQLSESSVTLSPVDSLESPHTYVSDTTSSPMITSPGILQASPNPMLATAAPPAPVHAQHALSFSNLHEMQPLAHGASTVLPSVSQLLSHHHIVSPGSGSAGSLSRLHPVPVPADWMNRMEVNETQYNEMFGMVLAPAEGTHPGIAPQSRPPEGKHITTPREPLPPIVTFQLIPKGSIAQPAGAPQPQSTCPPAVAGPLPTMYQIPEMARLPSVAFPTAMMPQQDGQVAQTILPAYHPFPASVGKYPTPPSQHSYASSNAAERTPSHSGHLQGEHPYLTPSPESPDQWSSSSPHSASDWSDVTTSPTPGGAGGGQRGPGTHMSEPPHNNMQVYA",
# ACHE_HUMAN
"MARAPLGVLLLLGLLGRGVGKNEELRLYHHLFNNYDPGSRPVREPEDTVTISLKVTLTNLISLNEKEETLTTSVWIGIDWQDYRLNYSKDDFGGIETLRVPSELVWLPEIVLENNIDGQFGVAYDANVLVYEGGSVTWLPPAIYRSVCAVEVTYFPFDWQNCSLIFRSQTYNAEEVEFTFAVDNDGKTINKIDIDTEAYTENGEWAIDFCPGVIRRHHGGATDGPGETDVIYSLIIRRKPLFYVINIIVPCVLISGLVLLAYFLPAQAGGQKCTVSINVLLAQTVFLFLIAQKIPETSLSVPLLGRFLIFVMVVATLIVMNCVIVLNVSQRTPTTHAMSPRLRHVLLELLPRLLGSPPPPEAPRAASPPRRASSVGLLLRAEELILKKPRSELVFEGQRHRQGTWTAAFCQSLGAAAPEVRCCVDAVNFVAESTRDQEATGEEVSDWVRMGNALDNICFWAALVLFSVGSSLIFLGAYFNRVPDLPYAPCIQP",
# 1433F_HUMAN
"MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLSVAYKNVVGARRSSWRVISSIEQKTMADGNEKKLEKVKAYREKIEKELETVCNDVLSLLDKFLIKNCNDFQYESKVFYLKMKGDYYRYLAEVASGEKKNSVVEASEAAYKEAFEISKEQMQPTHPIRLGLALNFSVFYYEIQNAPEQACLLAKQAFDDAIAELDTLNEDSYKDSTLIMQLLRDNLTLWTSDQQDEEAGEGN",
# UBE3A_HUMAN
"MEKLHQCYWKSGEPQSDDIEASRMKRAAAKHLIERYYHQLTEGCGNEACTNEFCASCPTFLRMDNNAAAIKALELYKINAKLCDPHPSKKGASSAYLENSKGAPNNSCSEIKMNKKGARIDFKDVTYLTEEKVYEILELCREREDYSPLIRVIGRVFSSAEALVQSFRKVKQHTKEELKSLQAKDEDKDEDEKEKAACSAAAMEEDSEASSSRIGDSSQGDNNLQKLGPDDVSVDIDAIRRVYTRLLSNEKIETAFLNALVYLSPNVECDLTYHNVYSRDPNYLNLFIIVMENRNLHSPEYLEMALPLFCKAMSKLPLAAQGKLIRLWSKYNADQIRRMMETFQQLITYKVISNEFNSRNLVNDDDAIVAASKCLKMVYYANVVGGEVDTNHNEEDDEEPIPESSELTLQELLGEERRNKKGPRVDPLETELGVKTLDCRKPLIPFEEFINEPLNEVLEMDKDYTFFKVETENKFSFMTCPFILNAVTKNLGLYYDNRIRMYSERRITVLYSLVQGQQLNPYLRLKVRRDHIIDDALVRLEMIAMENPADLKKQLYVEFEGEQGVDEGGVSKEFFQLVVEEIFNPDIGMFTYDESTKLFWFNPSSFETEGQFTLIGIVLGLAIYNNCILDVHFPMVVYRKLMGKKGTFRDLGDSHPVLYQSLKDLLEYEGNVEDDMMITFQISQTDLFGNPMMYDLKENGDKIPITNENRKEFVNLYSDYILNKSVEKQFKAFRRGFHMVTNESPLKYLFRPEEIELLICGSRNLDFQALEETTEYDGGYTRDSVLIREFWEIVHSFTDEQKRLFLQFTTGTDRAPVGGLGKLKMIIAKNGPDTERLPTSHTCFNVLLLPEYSSKEKLKERLLKAITYAKGFGML",
# DYN1_HUMAN
"MGNRGMEDLIPLVNRLQDAFSAIGQNADLDLPQIAVVGGQSAGKSSVLENFVGRDFLPRGSGIVTRRPLVLQLVNATTEYAEFLHCKGKKFTDFEEVRLEIEAETDRVTGTNKGISPVPINLRVYSPHVLNLTLVDLPGMTKVPVGDQPPDIEFQIRDMLMQFVTKENCLILAVSPANSDLANSDALKVAKEVDPQGQRTIGVITKLDLMDEGTDARDVLENKLLPLRRGYIGVVNRSQKDIDGKKDITAALAAERKFFLSHPSYRHLADRMGTPYLQKVLNQQLTNHIRDTLPGLRNKLQSQLLSIEKEVEEYKNFRPDDPARKTKALLQMVQQFAVDFEKRIEGSGDQIDTYELSGGARINRIFHERFPFELVKMEFDEKELRREISYAIKNIHGIRTGLFTPDMAFETIVKKQVKKIREPCLKCVDMVISELISTVRQCTKKLQQYPRLREEMERIVTTHIREREGRTKEQVMLLIDIELAYMNTNHEDFIGFANAQQRSNQMNKKKTSGNQDEILVIRKGWLTINNIGIMKGGSKEYWFVLTAENLSWYKDDEEKEKKYMLSVDNLKLRDVEKGFMSSKHIFALFNTEQRNVYKDYRQLELACETQEEVDSWKASFLRAGVYPERVGDKEKASETEENGSDSFMHSMDPQLERQVETIRNLVDSYMAIVNKTVRDLMPKTIMHLMINNTKEFIFSELLANLYSCGDQNTLMEESAEQAQRRDEMLRMYHALKEALSIIGDINTTTVSTPMPPPVDDSWLQVQSVPAGRRSPTSSPTPQRRAPAVPPARPGSRGPAPGPPPAGSALGGAPPVPSRPGASPDPFGPPPQVPSRPNRAPPGVPSRSGQASPSRPESPRPPFDL",
# DCE2_HUMAN
"MASPGSGFWSFGSEDGSGDSENPGTARAWCQVAQKFTGGIGNKLCALLYGDAEKPAESGGSQPPRAAARKAACACDQKPCSCSKVDVNYAFLHATDLLPACDGERPTLAFLQDVMNILLQYVVKSFDRSTKVIDFHYPNELLQEYNWELADQPQNLEEILMHCQTTLKYAIKTGHPRYFNQLSTGLDMVGLAADWLTSTANTNMFTYEIAPVFVLLEYVTLKKMREIIGWPGGSGDGIFSPGGAISNMYAMMIARFKMFPEVKEKGMAALPRLIAFTSEHSHFSLKKGAAALGIGTDSVILIKCDERGKMIPSDLERRILEAKQKGFVPFLVSATAGTTVYGAFDPLLAVADICKKYKIWMHVDAAWGGGLLMSRKHKWKLSGVERANSVTWNPHKMMGVPLQCSALLVREEGLMQNCNQMHASYLFQQDKHYDLSYDTGDKALQCGRHVDVFKLWLMWRAKGTTGFEAHVDKCLELAEYLYNIIKNREGYEMVFDGKPQHTNVCFWYIPPSLRTLEDNEERMSRLSKVAPVIKARMMEYGTTMVSYQPLGDKVNFFRMVISNPAATHQDIDFLIEEIERLGQDL",
# FAK1_HUMAN
"MAAAYLDPNLNHTPNSSTKTHLGTGMERSPGAMERVLKVFHYFESNSEPTTWASIIRHGDATDVRGIIQKIVDSHKVKHVACYGFRLSHLRSEEVHWLHVDMGVSSVREKYELAHPPEEWKYELRIRYLPKGFLNQFTEDKPTLNFFYQQVKSDYMLEIADQVDQEIALKLGCLEIRRSYWEMRGNALEKKSNYEVLEKDVGLKRFFPKSLLDSVKAKTLRKLIQQTFRQFANLNREESILKFFEILSPVYRFDKECFKCALGSSWIISVELAIGPEEGISYLTDKGCNPTHLADFTQVQTIQYSNSEDKDRKGMLQLKIAGAPEPLTVTAPSLTIAENMADLIDGYCRLVNGTSQSFIIRPQKEGERALPSIPKLANSEKQGMRTHAVSVSETDDYAEIIDEEDTYTMPSTRDYEIQRERIELGRCIGEGQFGDVHQGIYMSPENPALAVAIKTCKNCTSDSVREKFLQEALTMRQFDHPHIVKLIGVITENPVWIIMELCTLGELRSFLQVRKYSLDLASLILYAYQLSTALAYLESKRFVHRDIAARNVLVSSNDCVKLGDFGLSRYMEDSTYYKASKGKLPIKWMAPESINFRRFTSASDVWMFGVCMWEILMHGVKPFQGVKNNDVIGRIENGERLPMPPNCPPTLYSLMTKCWAYDPSRRPRFTELKAQLSTILEEEKAQQEERMRMESRRQATVSWDSGGSDEAPPKPSRPGYPSPRSSEGFYPSPQHMVQTNHYQVSGYPGSHGITAMAGSIYPGQASLLDQTDSWNHRPQEIAMWQPNVEDSTVLDLRGIGQVLPTHLMEERLIRQQQEMEEDQRWLEKEERFLKPDVRLSRGSIDREDGSLQGPIGNQHIYQPVGKPDPAAPPKKPPRPGAPGHLGSLASLSSPADSYNEGVKLQPQEISPPPTANLDRSNDKVYENVTGLVKAVIEMSSKIQPAPPEEYVPMVKEVGLALRTLLATVDETIPLLPASTHREIEMAQKLLNSDLGELINKMKLAQQYVMTSLQQEYKKQMLTAAHALAVDAKNLLDVIDQARLKMLGQTRPH",
# KPCZ_HUMAN
"MPSRTGPKMEGSGGRVRLKAHYGGDIFITSVDAATTFEELCEEVRDMCRLHQQHPLTLKWVDSEGDPCTVSSQMELEEAFRLARQCRDEGLIIHVFPSTPEQPGLPCPGEDKSIYRRGARRWRKLYRANGHLFQAKRFNRRAYCGQCSERIWGLARQGYRCINCKLLVHKRCHGLVPLTCRKHMDSVMPSQEPPVDDKNEDADLPSEETDGIAYISSSRKHDSIKDDSEDLKPVIDGMDGIKISQGLGLQDFDLIRVIGRGSYAKVLLVRLKKNDQIYAMKVVKKELVHDDEDIDWVQTEKHVFEQASSNPFLVGLHSCFQTTSRLFLVIEYVNGGDLMFHMQRQRKLPEEHARFYAAEICIALNFLHERGIIYRDLKLDNVLLDADGHIKLTDYGMCKEGLGPGDTTSTFCGTPNYIAPEILRGEEYGFSVDWWALGVLMFEMMAGRSPFDIITDNPDMNTEDYLFQVILEKPIRIPRFLSVKASHVLKGFLNKDPKERLGCRPQTGFSDIKSHAFFRSIDWDLLEKKQALPPFQPQITDDYGLDNFDTQFTSEPVQLTPDDEDAIKRIDQSEFEGFEYINPLLLSTEESV",
# NMDZ1_HUMAN
"MSTMRLLTLALLFSCSVARAACDPKIVNIGAVLSTRKHEQMFREAVNQANKRHGSWKIQLNATSVTHKPNAIQMALSVCEDLISSQVYAILVSHPPTPNDHFTPTPVSYTAGFYRIPVLGLTTRMSIYSDKSIHLSFLRTVPPYSHQSSVWFEMMRVYSWNHIILLVSDDHEGRAAQKRLETLLEERESKAEKVLQFDPGTKNVTALLMEAKELEARVIILSASEDDAATVYRAAAMLNMTGSGYVWLVGEREISGNALRYAPDGILGLQLINGKNESAHISDAVGVVAQAVHELLEKENITDPPRGCVGNTNIWKTGPLFKRVLMSSKYADGVTGRVEFNEDGDRKFANYSIMNLQNRKLVQVGIYNGTHVIPNDRKIIWPGGETEKPRGYQMSTRLKIVTIHQEPFVYVKPTLSDGTCKEEFTVNGDPVKKVICTGPNDTSPGSPRHTVPQCCYGFCIDLLIKLARTMNFTYEVHLVADGKFGTQERVNNSNKKEWNGMMGELLSGQADMIVAPLTINNERAQYIEFSKPFKYQGLTILVKKEIPRSTLDSFMQPFQSTLWLLVGLSVHVVAVMLYLLDRFSPFGRFKVNSEEEEEDALTLSSAMWFSWGVLLNSGIGEGAPRSFSARILGMVWAGFAMIIVASYTANLAAFLVLDRPEERITGINDPRLRNPSDKFIYATVKQSSVDIYFRRQVELSTMYRHMEKHNYESAAEAIQAVRDNKLHAFIWDSAVLEFEASQKCDLVTTGELFFRSGFGIGMRKDSPWKQNVSLSILKSHENGFMEDLDKTWVRYQECDSRSNAPATLTFENMAGVFMLVAGGIVAGIFLIFIEIAYKRHKDARRKQMQLAFAAVNVWRKNLQDRKSGRAEPDPKKKATFRAITSTLASSFKRRRSSKDTSTGGGRGALQNQKDTVLPRRAIEREEGQLQLCSRHRES",
# EF1A2_HUMAN
"MGKEKTHINIVVIGHVDSGKSTTTGHLIYKCGGIDKRTIEKFEKEAAEMGKGSFKYAWVLDKLKAERERGITIDISLWKFETTKYYITIIDAPGHRDFIKNMITGTSQADCAVLIVAAGVGEFEAGISKNGQTREHALLAYTLGVKQLIVGVNKMDSTEPAYSEKRYDEIVKEVSAYIKKIGYNPATVPFVPISGWHGDNMLEPSPNMPWFKGWKVERKEGNASGVSLLEALDTILPPTRPTDKPLRLPLQDVYKIGGIGTVPVGRVETGILRPGMVVTFAPVNITTEVKSVEMHHEALSEALPGDNVGFNVKNVSVKDIRRGNVCGDSKSDPPQEAAQFTSQVIILNHPGQISAGYSPVIDCHTAHIACKFAELKEKIDRRSGKKLEDNPKSLKSGDAAIVEMVPGKPMCVESFSQYPPLGRFAVRDMRQTVAVGVIKNVEKKSGGAGKVTKSAQKAQKAGK",
# KPCD_HUMAN
"MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQKKPTMYPEWKSTFDAHIYEGRVIQIVLMRAAEEPVSEVTVGVSVLAERCKKNNGKAEFWLDLQPQAKVLMSVQYFLEDVDCKQSMRSEDEAKFPTMNRRGAIKQAKIHYIKNHEFIATFFGQPTFCSVCKDFVWGLNKQGYKCRQCNAAIHKKCIDKIIGRCTGTAANSRDTIFQKERFNIDMPHRFKVHNYMSPTFCDHCGSLLWGLVKQGLKCEDCGMNVHHKCREKVANLCGINQKLLAEALNQVTQRASRRSDSASSEPVGIYQGFEKKTGVAGEDMQDNSGTYGKIWEGSSKCNINNFIFHKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVEKRVLTLAAENPFLTHLICTFQTKDHLFFVMEFLNGGDLMYHIQDKGRFELYRATFYAAEIMCGLQFLHSKGIIYRDLKLDNVLLDRDGHIKIADFGMCKENIFGESRASTFCGTPDYIAPEILQGLKYTFSVDWWSFGVLLYEMLIGQSPFHGDDEDELFESIRVDTPHYPRWITKESKDILEKLFEREPTKRLGVTGNIKIHPFFKTINWTLLEKRRLEPPFRPKVKSPRDYSNFDQEFLNEKARLSYSDKNLIDSMDQSAFAGFSFVNPKFEHLLED",
# CALD1_HUMAN
"MDDFERRRELRRQKREEMRLEAERIAYQRNDDDEEEAARERRRRARQERLRQKQEEESLGQVTDQVEVNAQNSVPDEEAKTTTTNTQVEGDDEAAFLERLARREERRQKRLQEALERQKEFDPTITDASLSLPSRRMQNDTAENETTEKEEKSESRQERYEIEETETVTKSYQKNDWRDAEENKKEDKEKEEEEEEKPKRGSIGENQVEVMVEEKTTESQEETVVMSLKNGQISSEEPKQEEEREQGSDEISHHEKMEEEDKERAEAERARLEAEERERIKAEQDKKIADERARIEAEEKAAAQERERREAEERERMREEEKRAAEERQRIKEEEKRAAEERQRIKEEEKRAAEERQRIKEEEKRAAEERQRARAEEEEKAKVEEQKRNKQLEEKKHAMQETKIKGEKVEQKIEGKWVNEKKAQEDKLQTAVLKKQGEEKGTKVQAKREKLQEDKPTFKKEEIKDEKIKKDKEPKEEVKSFMDRKKGFTEVKSQNGEFMTHKLKHTENTFSRPGGRASVDTKEAEGAPQVEAGKRLEELRRRRGETESEEFEKLKQKQQEAALELEELKKKREERRKVLEEEEQRRKQEEADRKLREEEEKRRLKEEIERRRAEAAEKRQKMPEDGLSDDKKPFKCFTPKGSSLKIEERAEFLNKSVQKSSGVKSTHQAAIVSKIDSRLEQYTSAIEGTKSAKPTKPAASDLPVPAEGVRNIKSMWEKGNVFSSPTAAGTPNKETAGLKVGVSSRINEWLTKTPDGNKSPAPKPSDLRPGDVSSKRNLWEKQSVDKVTSPTKV",
# ACHB3_HUMAN
"MLPDFMLVLIVLGIPSSATTGFNSIAENEDALLRHLFQGYQKWVRPVLHSNDTIKVYFGLKISQLVDVDEKNQLMTTNVWLKQEWTDHKLRWNPDDYGGIHSIKVPSESLWLPDIVLFENADGRFEGSLMTKVIVKSNGTVVWTPPASYKSSCTMDVTFFPFDRQNCSMKFGSWTYDGTMVDLILINENVDRKDFFDNGEWEILNAKGMKGNRRDGVYSYPFITYSFVLRRLPLFYTLFLIIPCLGLSFLTVLVFYLPSDEGEKLSLSTSVLVSLTVFLLVIEEIIPSSSKVIPLIGEYLLFIMIFVTLSIIVTVFVINVHHRSSSTYHPMAPWVKRLFLQKLPKLLCMKDHVDRYSSPEKEESQPVVKGKVLEKKKQKQLSDGEKVLVAFLEKAADSIRYISRHVKKEHFISQVVQDWKFVAQVLDRIFLWLFLIVSVTGSVLIFTPALKMWLHSYH",
# VMAT2_HUMAN
"MALSELALVRWLQESRRSRKLILFIVFLALLLDNMLLTVVVPIIPSYLYSIKHEKNATEIQTARPVHTASISDSFQSIFSYYDNSTMVTGNATRDLTLHQTATQHMVTNASAVPSDCPSEDKDLLNENVQVGLLFASKATVQLITNPFIGLLTNRIGYPIPIFAGFCIMFVSTIMFAFSSSYAFLLIARSLQGIGSSCSSVAGMGMLASVYTDDEERGNVMGIALGGLAMGVLVGPPFGSVLYEFVGKTAPFLVLAALVLLDGAIQLFVLQPSRVQPESQKGTPLTTLLKDPYILIAAGSICFANMGIAMLEPALPIWMMETMCSRKWQLGVAFLPASISYLIGTNIFGILAHKMGRWLCALLGMIIVGVSILCIPFAKNIYGLIAPNFGVGFAIGMVDSSMMPIMGYLVDLRHVSVYGSVYAIADVAFCMGYAIGPSAGGAIAKAIGFPWLMTIIGIIDILFAPLCFFLRSPPAKEEKMAILMDHNCPIKTKMYTQNNIQSYPIGEDEESESD",
# APLP2_HUMAN
"MAATGTAAAAATGRLLLLLLVGLTAPALALAGYIEALAANAGTGFAVAEPQIAMFCGKLNMHVNIQTGKWEPDPTGTKSCFETKEEVLQYCQEMYPELQITNVMEANQRVSIDNWCRRDKKQCKSRFVTPFKCLVGEFVSDVLLVPEKCQFFHKERMEVCENHQHWHTVVKEACLTQGMTLYSYGMLLPCGVDQFHGTEYVCCPQTKIIGSVSKEEEEEDEEEEEEEDEEEDYDVYKSEFPTEADLEDFTEAAVDEDDEDEEEGEEVVEDRDYYYDTFKGDDYNEENPTEPGSDGTMSDKEITHDVKAVCSQEAMTGPCRAVMPRWYFDLSKGKCVRFIYGGCGGNRNNFESEDYCMAVCKAMIPPTPLPTNDVDVYFETSADDNEHARFQKAKEQLEIRHRNRMDRVKKEWEEAELQAKNLPKAERQTLIQHFQAMVKALEKEAASEKQQLVETHLARVEAMLNDRRRMALENYLAALQSDPPRPHRILQALRRYVRAENKDRLHTIRHYQHVLAVDPEKAAQMKSQVMTHLHVIEERRNQSLSLLYKVPYVAQEIQEEIDELLQEQRADMDQFTASISETPVDVRVSSEESEEIPPFHPFHPFPALPENEDTQPELYHPMKKGSGVGEQDGGLIGAEEKVINSKNKVDENMVIDETLDVKEMIFNAERVGGLEEERESVGPLREDFSLSSSALIGLLVIAVAIATVIVISLVMLRKRQYGTISHGIVEVDPMLTPEERHLNKMQNHGYENPTYKYLEQMQI",
# FMR1_HUMAN
"MEELVVEVRGSNGAFYKAFVKDVHEDSITVAFENNWQPDRQIPFHDVRFPPPVGYNKDINESDEVEVYSRANEKEPCCWWLAKVRMIKGEFYVIEYAACDATYNEIVTIERLRSVNPNKPATKDTFHKIKLDVPEDLRQMCAKEAAHKDFKKAVGAFSVTYDPENYQLVILSINEVTSKRAHMLIDMHFRSLRTKLSLIMRNEEASKQLESSRQLASRFHEQFIVREDLMGLAIGTHGANIQQARKVPGVTAIDLDEDTCTFHIYGEDQDAVKKARSFLEFAEDVIQVPRNLVGKVIGKNGKLIQEIVDKSGVVRVRIEAENEKNVPQEEEIMPPNSLPSNNSRVGPNAPEEKKHLDIKENSTHFSQPNSTKVQRVLVASSVVAGESQKPELKAWQGMVPFVFVGTKDSIANATVLLDYHLNYLKEVDQLRLERLQIDEQLRQIGASSRPPPNRTDKEKSYVTDDGQGMGRGSRPYRNRGHGRRGPGYTSGTNSEASNASETESDHRDELSDWSLAPTEEERESFLRRGDGRRRGGGGRGQGGRGRGGGFKGNDDHSRTDNRPRNPREAKGRTTDGSLQIRVDCNNERSVHTKTLQNTSSEGSRLRTGKDRNQKKEKPDSVDGQQPLVNGVP",
# ACHD_HUMAN
"MEGPVLTLGLLAALAVCGSWGLNEEERLIRHLFQEKGYNKELRPVAHKEESVDVALALTLSNLISLKEVEETLTTNVWIEHGWTDNRLKWNAEEFGNISVLRLPPDMVWLPEIVLENNNDGSFQISYSCNVLVYHYGFVYWLPPAIFRSSCPISVTYFPFDWQNCSLKFSSLKYTAKEITLSLKQDAKENRTYPVEWIIIDPEGFTENGEWEIVHRPARVNVDPRAPLDSPSRQDITFYLIIRRKPLFYIINILVPCVLISFMVNLVFYLPADSGEKTSVAISVLLAQSVFLLLISKRLPATSMAIPLIGKFLLFGMVLVTMVVVICVIVLNIHFRTPSTHVLSEGVKKLFLETLPELLHMSRPAEDGPSPGALVRRSSSLGYISKAEEYFLLKSRSDLMFEKQSERHGLARRLTTARRPPASSEQAQQELFNELKPAVDGANFIVNHMRDQNNYNEEKDSWNRVARTVDRLCLFVVTPVMVVGTAWIFLQGVYNQPPPQPFPGDPYSYNVQDKRFI",
# RL18_HUMAN
"MGVDIRHNKDRKVRRKEPKSQDIYLRLLVKLYRFLARRTNSTFNQVVLKRLFMSRTNRPPLSLSRMIRKMKLPGRENKTAVVVGTITDDVRVQEVPKLKVCALRVTSRARSRILRAGGKILTFDQLALDSPKGCGTVLLSGPRKGREVYRHFGKAPGTPHSHTKPYVRSKGRKFERARGRRASRGYKN",
# C1QBP_HUMAN
"MLPLLRCVPRVLGSSVAGLRAAAPASPFRQLLQPAPRLCTRPFGLLSVRAGSERRPGLLRPRGPCACGCGCGSLHTDGDKAFVDFLSDEIKEERKIQKHKTLPKMSGGWELELNGTEAKLVRKVAGEKITVTFNINNSIPPTFDGEEEPSQGQKVEEQEPELTSTPNFVVEVIKNDDGKKALVLDCHYPEDEVGQEDEAESDIFSIREVSFQSTGESEWKDTNYTLNTDSLDWALYDHLMDFLADRGVDNTFADELVELSTALEHQEYITFLEDLKSFVKSQ",
# B2CL1_HUMAN
"MSQSNRELVVDFLSYKLSQKGYSWSQFSDVEENRTEAPEGTESEMETPSAINGNPSWHLADSPAVNGATGHSSSLDAREVIPMAAVKQALREAGDEFELRYRRAFSDLTSQLHITPGTAYQSFEQVVNELFRDGVNWGRIVAFFSFGGALCVESVDKEMQVLVSRIAAWMATYLNDHLEPWIQENGGWDTFVELYGNNAAAESRKGQERFNRWFLTGMTVAGVVLLGSLFSRK",
# SOS1_HUMAN
"MQAQQLPYEFFSEENAPKWRGLLVPALKKVQGQVHPTLESNDDALQYVEELILQLLNMLCQAQPRSASDVEERVQKSFPHPIDKWAIADAQSAIEKRKRRNPLSLPVEKIHPLLKEVLGYKIDHQVSVYIVAVLEYISADILKLVGNYVRNIRHYEITKQDIKVAMCADKVLMDMFHQDVEDINILSLTDEEPSTSGEQTYYDLVKAFMAEIRQYIRELNLIIKVFREPFVSNSKLFSANDVENIFSRIVDIHELSVKLLGHIEDTVEMTDEGSPHPLVGSCFEDLAEELAFDPYESYARDILRPGFHDRFLSQLSKPGAALYLQSIGEGFKEAVQYVLPRLLLAPVYHCLHYFELLKQLEEKSEDQEDKECLKQAITALLNVQSGMEKICSKSLAKRRLSESACRFYSQQMKGKQLAIKKMNEIQKNIDGWEGKDIGQCCNEFIMEGTLTRVGAKHERHIFLFDGLMICCKSNHGQPRLPGASNAEYRLKEKFFMRKVQINDKDDTNEYKHAFEIILKDENSVIFSAKSAEEKNNWMAALISLQYRSTLERMLDVTMLQEEKEEQMRLPSADVYRFAEPDSEENIIFEENMQPKAGIPIIKAGTVIKLIERLTYHMYADPNFVRTFLTTYRSFCKPQELLSLIIERFEIPEPEPTEADRIAIENGDQPLSAELKRFRKEYIQPVQLRVLNVCRHWVEHHFYDFERDAYLLQRMEEFIGTVRGKAMKKWVESITKIIQRKKIARDNGPGHNITFQSSPPTVEWHISRPGHIETFDLLTLHPIEIARQLTLLESDLYRAVQPSELVGSVWTKEDKEINSPNLLKMIRHTTNLTLWFEKCIVETENLEERVAVVSRIIEILQVFQELNNFNGVLEVVSAMNSSPVYRLDHTFEQIPSRQKKILEEAHELSEDHYKKYLAKLRSINPPCVPFFGIYLTNILKTEEGNPEVLKRHGKELINFSKRRKVAEITGEIQQYQNQPYCLRVESDIKRFFENLNPMGNSMEKEFTDYLFNKSLEIEPRNPKPLPRFPKKYSYPLKSPGVRPSNPRPGTMRHPTPLQQEPRKISYSRIPESETESTASAPNSPRTPLTPPPASGASSTTDVCSVFDSDHSSPFHSSNDTVFIQVTLPHGPRSASVSSISLTKGTDEVPVPPPVPPRRRPESAPAESSPSKIMSKHLDSPPAIPPRQPTSKAYSPRYSISDRTSISDPPESPPLLPPREPVRTPDVFSSSPLHLQPPPLGKKSDHGNAFFPNSPSPFTPPPPQTPSPHGTRRHLPSPPLTQEVDLHSIAGPPVPPRQSTSQHIPKLPPKTYKREHTHPSMHRDGPPLLENAHSS",
# ACK1_HUMAN
"MQPEEGTGWLLELLSEVQLQQYFLRLRDDLNVTRLSHFEYVKNEDLEKIGMGRPGQRRLWEAVKRRKALCKRKSWMSKVFSGKRLEAEFPPHHSQSTFRKTSPAPGGPAGEGPLQSLTCLIGEKDLRLLEKLGDGSFGVVRRGEWDAPSGKTVSVAVKCLKPDVLSQPEAMDDFIREVNAMHSLDHRNLIRLYGVVLTPPMKMVTELAPLGSLLDRLRKHQGHFLLGTLSRYAVQVAEGMGYLESKRFIHRDLAARNLLLATRDLVKIGDFGLMRALPQNDDHYVMQEHRKVPFAWCAPESLKTRTFSHASDTWMFGVTLWEMFTYGQEPWIGLNGSQILHKIDKEGERLPRPEDCPQDIYNVMVQCWAHKPEDRPTFVALRDFLLEAQPTDMRALQDFEEPDKLHIQMNDVITVIEGRAENYWWRGQNTRTLCVGPFPRNVVTSVAGLSAQDISQPLQNSFIHTGHGDSDPRHCWGFPDRIDELYLGNPMDPPDLLSVELSTSRPPQHLGGVKKPTYDPVSEDQDPLSSDFKRLGLRKPGLPRGLWLAKPSARVPGTKASRGSGAEVTLIDFGEEPVVPALRPCAPSLAQLAMDACSLLDETPPQSPTRALPRPLHPTPVVDWDARPLPPPPAYDDVAQDEDDFEICSINSTLVGAGVPAGPSQGQTNYAFVPEQARPPPPLEDNLFLPPQGGGKPPSSAQTAEIFQALQQECMRQLQAPAGSPAPSPSPGGDDKPQVPPRVPIPPRPTRPHVQLSPAPPGEEETSQWPGPASPPRVPPREPLSPQGSRTPSPLVPPGSSPLPPRLSSSPGKTMPTTQSFASDPKYATPQVIQAPGPRAGPCILPIVRDGKKVSSTHYYLLPERPSYLERYQRFLREAQSPEEPTPLPVPLLLPPPSTPAPAAPTATVRPMPQAALDPKANFSTNNSNPGARPPPPRATARLPQRGCPGDGPEAGRPADKIQMAMVHGVTTEECQAALQCHGWSVQRAAQYLKVEQLFGLGLRPRGECHKVLEMFDWNLEQAGCHLLGSWGPAHHKR",
# LRP1_HUMAN
"MLTPPLLLLLPLLSALVAAAIDAPKTCSPKQFACRDQITCISKGWRCDGERDCPDGSDEAPEICPQSKAQRCQPNEHNCLGTELCVPMSRLCNGVQDCMDGSDEGPHCRELQGNCSRLGCQHHCVPTLDGPTCYCNSSFQLQADGKTCKDFDECSVYGTCSQLCTNTDGSFICGCVEGYLLQPDNRSCKAKNEPVDRPPVLLIANSQNILATYLSGAQVSTITPTSTRQTTAMDFSYANETVCWVHVGDSAAQTQLKCARMPGLKGFVDEHTINISLSLHHVEQMAIDWLTGNFYFVDDIDDRIFVCNRNGDTCVTLLDLELYNPKGIALDPAMGKVFFTDYGQIPKVERCDMDGQNRTKLVDSKIVFPHGITLDLVSRLVYWADAYLDYIEVVDYEGKGRQTIIQGILIEHLYGLTVFENYLYATNSDNANAQQKTSVIRVNRFNSTEYQVVTRVDKGGALHIYHQRRQPRVRSHACENDQYGKPGGCSDICLLANSHKARTCRCRSGFSLGSDGKSCKKPEHELFLVYGKGRPGIIRGMDMGAKVPDEHMIPIENLMNPRALDFHAETGFIYFADTTSYLIGRQKIDGTERETILKDGIHNVEGVAVDWMGDNLYWTDDGPKKTISVARLEKAAQTRKTLIEGKMTHPRAIVVDPLNGWMYWTDWEEDPKDSRRGRLERAWMDGSHRDIFVTSKTVLWPNGLSLDIPAGRLYWVDAFYDRIETILLNGTDRKIVYEGPELNHAFGLCHHGNYLFWTEYRSGSVYRLERGVGGAPPTVTLLRSERPPIFEIRMYDAQQQQVGTNKCRVNNGGCSSLCLATPGSRQCACAEDQVLDADGVTCLANPSYVPPPQCQPGEFACANSRCIQERWKCDGDNDCLDNSDEAPALCHQHTCPSDRFKCENNRCIPNRWLCDGDNDCGNSEDESNATCSARTCPPNQFSCASGRCIPISWTCDLDDDCGDRSDESASCAYPTCFPLTQFTCNNGRCININWRCDNDNDCGDNSDEAGCSHSCSSTQFKCNSGRCIPEHWTCDGDNDCGDYSDETHANCTNQATRPPGGCHTDEFQCRLDGLCIPLRWRCDGDTDCMDSSDEKSCEGVTHVCDPSVKFGCKDSARCISKAWVCDGDNDCEDNSDEENCESLACRPPSHPCANNTSVCLPPDKLCDGNDDCGDGSDEGELCDQCSLNNGGCSHNCSVAPGEGIVCSCPLGMELGPDNHTCQIQSYCAKHLKCSQKCDQNKFSVKCSCYEGWVLEPDGESCRSLDPFKPFIIFSNRHEIRRIDLHKGDYSVLVPGLRNTIALDFHLSQSALYWTDVVEDKIYRGKLLDNGALTSFEVVIQYGLATPEGLAVDWIAGNIYWVESNLDQIEVAKLDGTLRTTLLAGDIEHPRAIALDPRDGILFWTDWDASLPRIEAASMSGAGRRTVHRETGSGGWPNGLTVDYLEKRILWIDARSDAIYSARYDGSGHMEVLRGHEFLSHPFAVTLYGGEVYWTDWRTNTLAKANKWTGHNVTVVQRTNTQPFDLQVYHPSRQPMAPNPCEANGGQGPCSHLCLINYNRTVSCACPHLMKLHKDNTTCYEFKKFLLYARQMEIRGVDLDAPYYNYIISFTVPDIDNVTVLDYDAREQRVYWSDVRTQAIKRAFINGTGVETVVSADLPNAHGLAVDWVSRNLFWTSYDTNKKQINVARLDGSFKNAVVQGLEQPHGLVVHPLRGKLYWTDGDNISMANMDGSNRTLLFSGQKGPVGLAIDFPESKLYWISSGNHTINRCNLDGSGLEVIDAMRSQLGKATALAIMGDKLWWADQVSEKMGTCSKADGSGSVVLRNSTTLVMHMKVYDESIQLDHKGTNPCSVNNGDCSQLCLPTSETTRSCMCTAGYSLRSGQQACEGVGSFLLYSVHEGIRGIPLDPNDKSDALVPVSGTSLAVGIDFHAENDTIYWVDMGLSTISRAKRDQTWREDVVTNGIGRVEGIAVDWIAGNIYWTDQGFDVIEVARLNGSFRYVVISQGLDKPRAITVHPEKGYLFWTEWGQYPRIERSRLDGTERVVLVNVSISWPNGISVDYQDGKLYWCDARTDKIERIDLETGENREVVLSSNNMDMFSVSVFEDFIYWSDRTHANGSIKRGSKDNATDSVPLRTGIGVQLKDIKVFNRDRQKGTNVCAVANGGCQQLCLYRGRGQRACACAHGMLAEDGASCREYAGYLLYSERTILKSIHLSDERNLNAPVQPFEDPEHMKNVIALAFDYRAGTSPGTPNRIFFSDIHFGNIQQINDDGSRRITIVENVGSVEGLAYHRGWDTLYWTSYTTSTITRHTVDQTRPGAFERETVITMSGDDHPRAFVLDECQNLMFWTNWNEQHPSIMRAALSGANVLTLIEKDIRTPNGLAIDHRAEKLYFSDATLDKIERCEYDGSHRYVILKSEPVHPFGLAVYGEHIFWTDWVRRAVQRANKHVGSNMKLLRVDIPQQPMGIIAVANDTNSCELSPCRINNGGCQDLCLLTHQGHVNCSCRGGRILQDDLTCRAVNSSCRAQDEFECANGECINFSLTCDGVPHCKDKSDEKPSYCNSRRCKKTFRQCSNGRCVSNMLWCNGADDCGDGSDEIPCNKTACGVGEFRCRDGTCIGNSSRCNQFVDCEDASDEMNCSATDCSSYFRLGVKGVLFQPCERTSLCYAPSWVCDGANDCGDYSDERDCPGVKRPRCPLNYFACPSGRCIPMSWTCDKEDDCEHGEDETHCNKFCSEAQFECQNHRCISKQWLCDGSDDCGDGSDEAAHCEGKTCGPSSFSCPGTHVCVPERWLCDGDKDCADGADESIAAGCLYNSTCDDREFMCQNRQCIPKHFVCDHDRDCADGSDESPECEYPTCGPSEFRCANGRCLSSRQWECDGENDCHDQSDEAPKNPHCTSQEHKCNASSQFLCSSGRCVAEALLCNGQDDCGDSSDERGCHINECLSRKLSGCSQDCEDLKIGFKCRCRPGFRLKDDGRTCADVDECSTTFPCSQRCINTHGSYKCLCVEGYAPRGGDPHSCKAVTDEEPFLIFANRYYLRKLNLDGSNYTLLKQGLNNAVALDFDYREQMIYWTDVTTQGSMIRRMHLNGSNVQVLHRTGLSNPDGLAVDWVGGNLYWCDKGRDTIEVSKLNGAYRTVLVSSGLREPRALVVDVQNGYLYWTDWGDHSLIGRIGMDGSSRSVIVDTKITWPNGLTLDYVTERIYWADAREDYIEFASLDGSNRHVVLSQDIPHIFALTLFEDYVYWTDWETKSINRAHKTTGTNKTLLISTLHRPMDLHVFHALRQPDVPNHPCKVNNGGCSNLCLLSPGGGHKCACPTNFYLGSDGRTCVSNCTASQFVCKNDKCIPFWWKCDTEDDCGDHSDEPPDCPEFKCRPGQFQCSTGICTNPAFICDGDNDCQDNSDEANCDIHVCLPSQFKCTNTNRCIPGIFRCNGQDNCGDGEDERDCPEVTCAPNQFQCSITKRCIPRVWVCDRDNDCVDGSDEPANCTQMTCGVDEFRCKDSGRCIPARWKCDGEDDCGDGSDEPKEECDERTCEPYQFRCKNNRCVPGRWQCDYDNDCGDNSDEESCTPRPCSESEFSCANGRCIAGRWKCDGDHDCADGSDEKDCTPRCDMDQFQCKSGHCIPLRWRCDADADCMDGSDEEACGTGVRTCPLDEFQCNNTLCKPLAWKCDGEDDCGDNSDENPEECARFVCPPNRPFRCKNDRVCLWIGRQCDGTDNCGDGTDEEDCEPPTAHTTHCKDKKEFLCRNQRCLSSSLRCNMFDDCGDGSDEEDCSIDPKLTSCATNASICGDEARCVRTEKAAYCACRSGFHTVPGQPGCQDINECLRFGTCSQLCNNTKGGHLCSCARNFMKTHNTCKAEGSEYQVLYIADDNEIRSLFPGHPHSAYEQAFQGDESVRIDAMDVHVKAGRVYWTNWHTGTISYRSLPPAAPPTTSNRHRRQIDRGVTHLNISGLKMPRGIAIDWVAGNVYWTDSGRDVIEVAQMKGENRKTLISGMIDEPHAIVVDPLRGTMYWSDWGNHPKIETAAMDGTLRETLVQDNIQWPTGLAVDYHNERLYWADAKLSVIGSIRLNGTDPIVAADSKRGLSHPFSIDVFEDYIYGVTYINNRVFKIHKFGHSPLVNLTGGLSHASDVVLYHQHKQPEVTNPCDRKKCEWLCLLSPSGPVCTCPNGKRLDNGTCVPVPSPTPPPDAPRPGTCNLQCFNGGSCFLNARRQPKCRCQPRYTGDKCELDQCWEHCRNGGTCAASPSGMPTCRCPTGFTGPKCTQQVCAGYCANNSTCTVNQGNQPQCRCLPGFLGDRCQYRQCSGYCENFGTCQMAADGSRQCRCTAYFEGSRCEVNKCSRCLEGACVVNKQSGDVTCNCTDGRVAPSCLTCVGHCSNGGSCTMNSKMMPECQCPPHMTGPRCEEHVFSQQQPGHIASILIPLLLLLLLVLVAGVVFWYKRRVQGAKGFQHQRMTNGAMNVEIGNPTYKMYEGGEPDDVGGLLDADFALDPDKPTNFTNPVYATLYMGGHGSRHSLASTDEKRELLGRGPEDEIGDPLA",
# FOXM1_HUMAN
"MKTSPRRPLILKRRRLPLPVQNAPSETSEEEPKRSPAQQESNQAEASKEVAESNSCKFPAGIKIINHPTMPNTQVVAIPNNANIHSIITALTAKGKESGSSGPNKFILISCGGAPTQPPGLRPQTQTSYDAKRTEVTLETLGPKPAARDVNLPRPPGALCEQKRETCADGEAAGCTINNSLSNIQWLRKMSSDGLGSRSIKQEMEEKENCHLEQRQVKVEEPSRPSASWQNSVSERPPYSYMAMIQFAINSTERKRMTLKDIYTWIEDHFPYFKHIAKPGWKNSIRHNLSLHDMFVRETSANGKVSFWTIHPSANRYLTLDQVFKPLDPGSPQLPEHLESQQKRPNPELRRNMTIKTELPLGARRKMKPLLPRVSSYLVPIQFPVNQSLVLQPSVKVPLPLAASLMSSELARHSKRVRIAPKVLLAEEGIAPLSSAGPGKEEKLLFGEGFSPLLPVQTIKEEEIQPGEEMPHLARPIKVESPPLEEWPSPAPSFKEESSHSWEDSSQSPTPRPKKSYSGLRSPTRCVSEMLVIQHRERRERSRSRRKQHLLPPCVDEPELLFSEGPSTSRWAAELPFPADSSDPASQLSYSQEVGGPFKTPIKETLPISSTPSKSVLPRTPESWRLTPPAKVGGLDFSPVQTSQGASDPLPDPLGLMDLSTTPLQSAPPLESPQRLLSSEPLDLISVPFGNSSPSDIDVPKPGSPEPQVSGLAANRSLTEGLVLDTMNDSLSKILLDISFPGLDEDPLGPDNINWSQFIPELQ",
# PP2BA_HUMAN
"MSEPKAIDPKLSTTDRVVKAVPFPPSHRLTAKEVFDNDGKPRVDILKAHLMKEGRLEESVALRIITEGASILRQEKNLLDIDAPVTVCGDIHGQFFDLMKLFEVGGSPANTRYLFLGDYVDRGYFSIECVLYLWALKILYPKTLFLLRGNHECRHLTEYFTFKQECKIKYSERVYDACMDAFDCLPLAALMNQQFLCVHGGLSPEINTLDDIRKLDRFKEPPAYGPMCDILWSDPLEDFGNEKTQEHFTHNTVRGCSYFYSYPAVCEFLQHNNLLSILRAHEAQDAGYRMYRKSQTTGFPSLITIFSAPNYLDVYNNKAAVLKYENNVMNIRQFNCSPHPYWLPNFMDVFTWSLPFVGEKVTEMLVNVLNICSDDELGSEEDGFDGATAAARKEVIRNKIRAIGKMARVFSVLREESESVLTLKGLTPTGMLPSGVLSGGKQTLQSATVEAIEADEAIKGFSPQHKITSFEEAKGLDRINERMPPRRDAMPSDANLNSINKALTSETNGTDSNGSNSSNIQ",
# CACB2_HUMAN
"MVQRDMSKSPPTAAAAVAQEIQMELLENVAPAGALGAAAQSYGKGARRKNRFKGSDGSTSSDTTSNSFVRQGSADSYTSRPSDSDVSLEEDREAVRREAERQAQAQLEKAKTKPVAFAVRTNVSYSAAHEDDVPVPGMAISFEAKDFLHVKEKFNNDWWIGRLVKEGCEIGFIPSPVKLENMRLQHEQRAKQGKFYSSKSGGNSSSSLGDIVPSSRKSTPPSSAIDIDATGLDAEENDIPANHRSPKPSANSVTSPHSKEKRMPFFKKTEHTPPYDVVPSMRPVVLVGPSLKGYEVTDMMQKALFDFLKHRFEGRISITRVTADISLAKRSVLNNPSKHAIIERSNTRSSLAEVQSEIERIFELARTLQLVVLDADTINHPAQLSKTSLAPIIVYVKISSPKVLQRLIKSRGKSQAKHLNVQMVAADKLAQCPPELFDVILDENQLEDACEHLADYLEAYWKATHPPSSSLPNPLLSRTLATSSLPLSPTLASNSQGSQGDQRTDRSAPIRSASQAEEEPSVEPVKKSQHRSSSSAPHHNHRSGTSRGLSRQETFDSETQESRDSAYVEPKEDYSHDHVDHYASHRDHNHRDETHGSSDHRHRESRHRSRDVDREQDHNECNKQRSRHKSKDRYCEKDGEVISKKRNEAGEWNRDVYIRQ",
# DEMA_HUMAN
"MERLQKQPLTSPGSVSPSRDSSVPGSPSSIVAKMDNQVLGYKDLAAIPKDKAILDIERPDLMIYEPHFTYSLLEHVELPRSRERSLSPKSTSPPPSPEVWADSRSPGIISQASAPRTTGTPRTSLPHFHHPETSRPDSNIYKKPPIYKQRESVGGSPQTKHLIEDLIIESSKFPAAQPPDPNQPAKIETDYWPCPPSLAVVETEWRKRKASRRGAEEEEEEEDDDSGEEMKALRERQREELSKVTSNLGKMILKEEMEKSLPIRRKTRSLPDRTPFHTSLHQGTSKSSSLPAYGRTTLSRLQSTEFSPSGSETGSPGLQNGEGQRGRMDRGNSLPCVLEQKIYPYEMLVVTNKGRTKLPPGVDRMRLERHLSAEDFSRVFAMSPEEFGKLALWKRNELKKKASLF",
# TICN1_HUMAN
"MPAIAVLAAAAAAWCFLQVESRHLDALAGGAGPNHGNFLDNDQWLSTVSQYDRDKYWNRFRDDDYFRNWNPNKPFDQALDPSKDPCLKVKCSPHKVCVTQDYQTALCVSRKHLLPRQKKGNVAQKHWVGPSNLVKCKPCPVAQSAMVCGSDGHSYTSKCKLEFHACSTGKSLATLCDGPCPCLPEPEPPKHKAERSACTDKELRNLASRLKDWFGALHEDANRVIKPTSSNTAQGRFDTSILPICKDSLGWMFNKLDMNYDLLLDPSEINAIYLDKYEPCIKPLFNSCDSFKDGKLSNNEWCYCFQKPGGLPCQNEMNRIQKLSKGKSLLGAFIPRCNEEGYYKATQCHGSTGQCWCVDKYGNELAGSRKQGAVSCEEEQETSGDFGSGGSVVLLDDLEYERELGPKDKEGKLRVHTRAVTEDDEDEDDDKEDEVGYIW",
# CD47_HUMAN
"MWPLVAALLLGSACCGSAQLLFNKTKSVEFTFCNDTVVIPCFVTNMEAQNTTEVYVKWKFKGRDIYTFDGALNKSTVPTDFSSAKIEVSQLLKGDASLKMDKSDAVSHTGNYTCEVTELTREGETIIELKYRVVSWFSPNENILIVIFPIFAILLFWGQFGIKTLKYRSGGMDEKTIALLVAGLVITVIVIVGAILFVPGEYSLKNATGLGLIVTSTGILILLHYYVFSTAIGLTSFVIAILVIQVIAYILAVVGLSLCIAACIPMHGPLLISGLSILALAQLLGLVYMKFVASNQKTIQPPRKAVEEPLNAFKESKGMMNDE",
# ADCY1_HUMAN
"MAGAPRGGGGGGGGAGEPGGAERAAGTSRRRGLRACDEEFACPELEALFRGYTLRLEQAATLKALAVLSLLAGALALAELLGAPGPAPGLAKGSHPVHCVLFLALLVVTNVRSLQVPQLQQVGQLALLFSLTFALLCCPFALGGPARGSAGAAGGPATAEQGVWQLLLVTFVSYALLPVRSLLAIGFGLVVAASHLLVTATLVPAKRPRLWRTLGANALLFVGVNMYGVFVRILTERSQRKAFLQARSCIEDRLRLEDENEKQERLLMSLLPRNVAMEMKEDFLKPPERIFHKIYIQRHDNVSILFADIVGFTGLASQCTAQELVKLLNELFGKFDELATENHCRRIKILGDCYYCVSGLTQPKTDHAHCCVEMGLDMIDTITSVAEATEVDLNMRVGLHTGRVLCGVLGLRKWQYDVWSNDVTLANVMEAAGLPGKVHITKTTLACLNGDYEVEPGYGHERNSFLKTHNIETFFIVPSHRRKIFPGLILSDIKPAKRMKFKTVCYLLVQLMHCRKMFKAEIPFSNVMTCEDDDKRRALRTASEKLRNRSSFSTNVVYTTPGTRVNRYISRLLEARQTELEMADLNFFTLKYKHVEREQKYHQLQDEYFTSAVVLTLILAALFGLVYLLIFPQSVVVLLLLVFCICFLVACVLYLHITRVQCFPGCLTIQIRTVLCIFIVVLIYSVAQGCVVGCLPWAWSSKPNSSLVVLSSGGQRTALPTLPCESTHHALLCCLVGTLPLAIFFRVSSLPKMILLSGLTTSYILVLELSGYTRTGGGAVSGRSYEPIVAILLFSCALALHARQVDIRLRLDYLWAAQAEEEREDMEKVKLDNRRILFNLLPAHVAQHFLMSNPRNMDLYYQSYSQVGVMFASIPNFNDFYIELDGNNMGVECLRLLNEIIADFDELMEKDFYKDIEKIKTIGSTYMAAVGLAPTSGTKAKKSISSHLSTLADFAIEMFDVLDEINYQSYNDFVLRVGINVGPVVAGVIGARRPQYDIWGNTVNVASRMDSTGVQGRIQVTEEVHRLLRRCPYHFVCRGKVSVKGKGEMLTYFLEGRTDGNGSQIRSLGLDRKMCPFGRAGLQGRRPPVCPMPGVSVRAGLPPHSPGQYLPSAAAGKEA",
# VAC14_HUMAN
"MNPEKDFAPLTPNIVRALNDKLYEKRKVAALEIEKLVREFVAQNNTVQIKHVIQTLSQEFALSQHPHSRKGGLIGLAACSIALGKDSGLYLKELIEPVLTCFNDADSRLRYYACEALYNIVKVARGAVLPHFNVLFDGLSKLAADPDPNVKSGSELLDRLLKDIVTESNKFDLVSFIPLLRERIYSNNQYARQFIISWILVLESVPDINLLDYLPEILDGLFQILGDNGKEIRKMCEVVLGEFLKEIKKNPSSVKFAEMANILVIHCQTTDDLIQLTAMCWMREFIQLAGRVMLPYSSGILTAVLPCLAYDDRKKSIKEVANVCNQSLMKLVTPEDDELDELRPGQRQAEPTPDDALPKQEGTASGGPDGSCDSSFSSGISVFTAASTERAPVTLHLDGIVQVLNCHLSDTAIGMMTRIAVLKWLYHLYIKTPRKMFRHTDSLFPILLQTLSDESDEVILKDLEVLAEIASSPAGQTDDPGPLDGPDLQASHSELQVPTPGRAGLLNTSGTKGLECSPSTPTMNSYFYKFMINLLKRFSSERKLLEVRGPFIIRQLCLLLNAENIFHSMADILLREEDLKFASTMVHALNTILLTSTELFQLRNQLKDLKTLESQNLFCCLYRSWCHNPVTTVSLCFLTQNYRHAYDLIQKFGDLEVTVDFLAEVDKLVQLIECPIFTYLRLQLLDVKNNPYLIKALYGLLMLLPQSSAFQLLSHRLQCVPNPELLQTEDSLKAAPKSQKADSPSIDYAELLQHFEKVQNKHLEVRHQRSGRGDHLDRRVVL",
# ABCC8_HUMAN
"MPLAFCGSENHSAAYRVDQGVLNNGCFVDALNVVPHVFLLFITFPILFIGWGSQSSKVHIHHSTWLHFPGHNLRWILTFMLLFVLVCEIAEGILSDGVTESHHLHLYMPAGMAFMAAVTSVVYYHNIETSNFPKLLIALLVYWTLAFITKTIKFVKFLDHAIGFSQLRFCLTGLLVILYGMLLLVEVNVIRVRRYIFFKTPREVKPPEDLQDLGVRFLQPFVNLLSKGTYWWMNAFIKTAHKKPIDLRAIGKLPIAMRALTNYQRLCEAFDAQVRKDIQGTQGARAIWQALSHAFGRRLVLSSTFRILADLLGFAGPLCIFGIVDHLGKENDVFQPKTQFLGVYFVSSQEFLANAYVLAVLLFLALLLQRTFLQASYYVAIETGINLRGAIQTKIYNKIMHLSTSNLSMGEMTAGQICNLVAIDTNQLMWFFFLCPNLWAMPVQIIVGVILLYYILGVSALIGAAVIILLAPVQYFVATKLSQAQRSTLEYSNERLKQTNEMLRGIKLLKLYAWENIFRTRVETTRRKEMTSLRAFAIYTSISIFMNTAIPIAAVLITFVGHVSFFKEADFSPSVAFASLSLFHILVTPLFLLSSVVRSTVKALVSVQKLSEFLSSAEIREEQCAPHEPTPQGPASKYQAVPLRVVNRKRPAREDCRGLTGPLQSLVPSADGDADNCCVQIMGGYFTWTPDGIPTLSNITIRIPRGQLTMIVGQVGCGKSSLLLAALGEMQKVSGAVFWSSLPDSEIGEDPSPERETATDLDIRKRGPVAYASQKPWLLNATVEENIIFESPFNKQRYKMVIEACSLQPDIDILPHGDQTQIGERGINLSGGQRQRISVARALYQHANVVFLDDPFSALDIHLSDHLMQAGILELLRDDKRTVVLVTHKLQYLPHADWIIAMKDGTIQREGTLKDFQRSECQLFEHWKTLMNRQDQELEKETVTERKATEPPQGLSRAMSSRDGLLQDEEEEEEEAAESEEDDNLSSMLHQRAEIPWRACAKYLSSAGILLLSLLVFSQLLKHMVLVAIDYWLAKWTDSALTLTPAARNCSLSQECTLDQTVYAMVFTVLCSLGIVLCLVTSVTVEWTGLKVAKRLHRSLLNRIILAPMRFFETTPLGSILNRFSSDCNTIDQHIPSTLECLSRSTLLCVSALAVISYVTPVFLVALLPLAIVCYFIQKYFRVASRDLQQLDDTTQLPLLSHFAETVEGLTTIRAFRYEARFQQKLLEYTDSNNIASLFLTAANRWLEVRMEYIGACVVLIAAVTSISNSLHRELSAGLVGLGLTYALMVSNYLNWMVRNLADMELQLGAVKRIHGLLKTEAESYEGLLAPSLIPKNWPDQGKIQIQNLSVRYDSSLKPVLKHVNALIAPGQKIGICGRTGSGKSSFSLAFFRMVDTFEGHIIIDGIDIAKLPLHTLRSRLSIILQDPVLFSGTIRFNLDPERKCSDSTLWEALEIAQLKLVVKALPGGLDAIITEGGENFSQGQRQLFCLARAFVRKTSIFIMDEATASIDMATENILQKVVMTAFADRTVVTIAHRVHTILSADLVIVLKRGAILEFDKPEKLLSRKDSVFASFVRADK",
# KCNA1_HUMAN
"MTVMSGENVDEASAAPGHPQDGSYPRQADHDDHECCERVVINISGLRFETQLKTLAQFPNTLLGNPKKRMRYFDPLRNEYFFDRNRPSFDAILYYYQSGGRLRRPVNVPLDMFSEEIKFYELGEEAMEKFREDEGFIKEEERPLPEKEYQRQVWLLFEYPESSGPARVIAIVSVMVILISIVIFCLETLPELKDDKDFTGTVHRIDNTTVIYNSNIFTDPFFIVETLCIIWFSFELVVRFFACPSKTDFFKNIMNFIDIVAIIPYFITLGTEIAEQEGNQKGEQATSLAILRVIRLVRVFRIFKLSRHSKGLQILGQTLKASMRELGLLIFFLFIGVILFSSAVYFAEAEEAESHFSSIPDAFWWAVVSMTTVGYGDMYPVTIGGKIVGSLCAIAGVLTIALPVPVIVSNFNYFYHRETEGEEQAQLLHVSSPNLASDSDLSRRSSSTMSKSEYMEIEEDMNNSIAHYRQVNIRTANCTTANQNCVNKSKLLTDV",
# FCHO2_HUMAN
"MVMAYFVENFWGEKNSGFDVLYHNMKHGQISTKELADFVRERATIEEAYSRSMTKLAKSASNYSQLGTFAPVWDVFKTSTEKLANCHLDLVRKLQELIKEVQKYGEEQVKSHKKTKEEVAGTLEAVQTIQSITQALQKSKENYNAKCVEQERLKKEGATQREIEKAAVKSKKATDTYKLYVEKYALAKADFEQKMTETAQKFQDIEETHLIHIKEIIGSLSNAIKEIHLQIGQVHEEFINNMANTTVESLIQKFAESKGTGKERPGLIEFEECDTASAVEGIKPRKRKTFALPGIIKKEKDAESVECPDADSLNIPDVDEEGYSIKPETNQNDTKENHFYSSSDSDSEDEEPKKYRIEIKPMHPNNSHHTMASLDELKVSIGNITLSPAISRHSPVQMNRNLSNEELTKSKPSAPPNEKGTSDLLAWDPLFGPSLDSSSSSSLTSSSSARPTTPLSVGTIVPPPRPASRPKLTSGKLSGINEIPRPFSPPVTSNTSPPPAAPLARAESSSSISSSASLSAANTPTVGVSRGPSPVSLGNQDTLPVAVALTESVNAYFKGADPTKCIVKITGDMTMSFPSGIIKVFTSNPTPAVLCFRVKNISRLEQILPNAQLVFSDPSQCDSNTKDFWMNMQAVTVYLKKLSEQNPAASYYNVDVLKYQVSSNGIQSTPLNLATYWKCSASTTDLRVDYKYNPEAMVAPSVLSNIQVVVPVDGGVTNMQSLPPAIWNAEQMKAFWKLSSISEKSENGGSGSLRAKFDLSEGPSKPTTLAVQFLSEGSTLSGVDFELVGTGYRLSLIKKRFATGRYLADC",
# AP1B1_HUMAN
"MTDSKYFTTTKKGEIFELKAELNSDKKEKKKEAVKKVIASMTVGKDVSALFPDVVNCMQTDNLELKKLVYLYLMNYAKSQPDMAIMAVNTFVKDCEDPNPLIRALAVRTMGCIRVDKITEYLCEPLRKCLKDEDPYVRKTAAVCVAKLHDINAQLVEDQGFLDTLKDLISDSNPMVVANAVAALSEIAESHPSSNLLDLNPQSINKLLTALNECTEWGQIFILDCLANYMPKDDREAQSICERVTPRLSHANSAVVLSAVKVLMKFMEMLSKDLDYYGTLLKKLAPPLVTLLSAEPELQYVALRNINLIVQKRPEILKHEMKVFFVKYNDPIYVKLEKLDIMIRLASQANIAQVLAELKEYATEVDVDFVRKAVRAIGRCAIKVEQSAERCVSTLLDLIQTKVNYVVQEAIVVIKDIFRKYPNKYESVIATLCENLDSLDEPEARAAMIWIVGEYAERIDNADELLESFLEGFHDESTQVQLQLLTAIVKLFLKKPTETQELVQQVLSLATQDSDNPDLRDRGYIYWRLLSTDPVAAKEVVLAEKPLISEETDLIEPTLLDELICYIGTLASVYHKPPSAFVEGGRGVVHKSLPPRTASSESAESPETAPTGAPPGEQPDVIPAQGDLLGDLLNLDLGPPVSGPPLATSSVQMGAVDLLGGGLDSLMGDEPEGIGGTNFVAPPTAAVPANLGAPIGSGLSDLFDLTSGVGTLSGSYVAPKAVWLPAMKAKGLEISGTFTRQVGSISMDLQLTNKALQVMTDFAIQFNRNSFGLAPAAPLQVHAPLSPNQTVEISLPLSTVGSVMKMEPLNNLQVAVKNNIDVFYFSTLYPLHILFVEDGKMDRQMFLATWKDIPNENEAQFQIRDCPLNAEAASSKLQSSNIFTVAKRNVEGQDMLYQSLKLTNGIWVLAELRIQPGNPSCTDLELSLKCRAPEVSQHVYQAYETILKN",
# KIF1A_HUMAN
"MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKETPKSFSFDYSYWSHTSPEDINYASQKQVYRDIGEEMLQHAFEGYNVCIFAYGQTGAGKSYTMMGKQEKDQQGIIPQLCEDLFSRINDTTNDNMSYSVEVSYMEIYCERVRDLLNPKNKGNLRVREHPLLGPYVEDLSKLAVTSYNDIQDLMDSGNKARTVAATNMNETSSRSHAVFNIIFTQKRHDAETNITTEKVSKISLVDLAGSERADSTGAKGTRLKEGANINKSLTTLGKVISALAEMDSGPNKNKKKKKTDFIPYRDSVLTWLLRENLGGNSRTAMVAALSPADINYDETLSTLRYADRAKQIRCNAVINEDPNNKLIRELKDEVTRLRDLLYAQGLGDITDMTNALVGMSPSSSLSALSSRAASVSSLHERILFAPGSEEAIERLKETEKIIAELNETWEEKLRRTEAIRMEREALLAEMGVAMREDGGTLGVFSPKKTPHLVNLNEDPLMSECLLYYIKDGITRVGREDGERRQDIVLSGHFIKEEHCVFRSDSRGGSEAVVTLEPCEGADTYVNGKKVTEPSILRSGNRIIMGKSHVFRFNHPEQARQERERTPCAETPAEPVDWAFAQRELLEKQGIDMKQEMEQRLQELEDQYRREREEATYLLEQQRLDYESKLEALQKQMDSRYYPEVNEEEEEPEDEVQWTERECELALWAFRKWKWYQFTSLRDLLWGNAIFLKEANAISVELKKKVQFQFVLLTDTLYSPLPPDLLPPEAAKDRETRPFPRTIVAVEVQDQKNGATHYWTLEKLRQRLDLMREMYDRAAEVPSSVIEDCDNVVTGGDPFYDRFPWFRLVGRAFVYLSNLLYPVPLVHRVAIVSEKGEVKGFLRVAVQAISADEEAPDYGSGVRQSGTAKISFDDQHFEKFQSESCPVVGMSRSGTSQEELRIVEGQGQGADVGPSADEVNNNTCSAVPPEGLLLDSSEKAALDGPLDAALDHLRLGNTFTFRVTVLQASSISAEYADIFCQFNFIHRHDEAFSTEPLKNTGRGPPLGFYHVQNIAVEVTKSFIEYIKSQPIVFEVFGHYQQHPFPPLCKDVLSPLRPSRRHFPRVMPLSKPVPATKLSTLTRPCPGPCHCKYDLLVYFEICELEANGDYIPAVVDHRGGMPCMGTFLLHQGIQRRITVTLLHETGSHIRWKEVRELVVGRIRNTPETDESLIDPNILSLNILSSGYIHPAQDDRTFYQFEAAWDSSMHNSLLLNRVTPYREKIYMTLSAYIEMENCTQPAVVTKDFCMVFYSRDAKLPASRSIRNLFGSGSLRASESNRVTGVYELSLCHVADAGSPGMQRRRRRVLDTSVAYVRGEENLAGWRPRSDSLILDHQWELEKLSLLQEVEKTRHYLLLREKLETAQRPVPEALSPAFSEDSESHGSSSASSPLSAEGRPSPLEAPNERQRELAVKCLRLLTHTFNREYTHSHVCVSASESKLSEMSVTLLRDPSMSPLGVATLTPSSTCPSLVEGRYGATDLRTPQPCSRPASPEPELLPEADSKKLPSPARATETDKEPQRLLVPDIQEIRVSPIVSKKGYLHFLEPHTSGWARRFVVVRRPYAYMYNSDKDTVERFVLNLATAQVEYSEDQQAMLKTPNTFAVCTEHRGILLQAASDKDMHDWLYAFNPLLAGTIRSKLSRRRSAQMRV",
# SCRN1_HUMAN
"MAAAPPSYCFVAFPPRAKDGLVVFGKNSARPRDEVQEVVYFSAADHEPESKVECTYISIDQVPRTYAIMISRPAWLWGAEMGANEHGVCIANEAINTREPAAEIEALLGMDLVRLGLERGETAKEALDVIVSLLEEHGQGGNYFEDANSCHSFQSAYLIVDRDEAWVLETIGKYWAAEKVTEGVRCICSQLSLTTKMDAEHPELRSYAQSQGWWTGEGEFNFSEVFSPVEDHLDCGAGKDSLEKQEESITVQTMMNTLRDKASGVCIDSEFFLTTASGVSVLPQNRSSPCIHYFTGTPDPSRSIFKPFIFVDDVKLVPKTQSPCFGDDDPAKKEPRFQEKPDRRHELYKAHEWARAIIESDQEQGRKLRSTMLELEKQGLEAMEEILTSSEPLDPAEVGDLFYDCVDTEIKFFK",
# KCMA1_HUMAN
"MANGGGGGGGSSGGGGGGGGSSLRMSSNIHANHLSLDASSSSSSSSSSSSSSSSSSSSSSVHEPKMDALIIPVTMEVPCDSRGQRMWWAFLASSMVTFFGGLFIILLWRTLKYLWTVCCHCGGKTKEAQKINNGSSQADGTLKPVDEKEEAVAAEVGWMTSVKDWAGVMISAQTLTGRVLVVLVFALSIGALVIYFIDSSNPIESCQNFYKDFTLQIDMAFNVFFLLYFGLRFIAANDKLWFWLEVNSVVDFFTVPPVFVSVYLNRSWLGLRFLRALRLIQFSEILQFLNILKTSNSIKLVNLLSIFISTWLTAAGFIHLVENSGDPWENFQNNQALTYWECVYLLMVTMSTVGYGDVYAKTTLGRLFMVFFILGGLAMFASYVPEIIELIGNRKKYGGSYSAVSGRKHIVVCGHITLESVSNFLKDFLHKDRDDVNVEIVFLHNISPNLELEALFKRHFTQVEFYQGSVLNPHDLARVKIESADACLILANKYCADPDAEDASNIMRVISIKNYHPKIRIITQMLQYHNKAHLLNIPSWNWKEGDDAICLAELKLGFIAQSCLAQGLSTMLANLFSMRSFIKIEEDTWQKYYLEGVSNEMYTEYLSSAFVGLSFPTVCELCFVKLKLLMIAIEYKSANRESRILINPGNHLKIQEGTLGFFIASDAKEVKRAFFYCKACHDDITDPKRIKKCGCKRPKMSIYKRMRRACCFDCGRSERDCSCMSGRVRGNVDTLERAFPLSSVSVNDCSTSFRAFEDEQPSTLSPKKKQRNGGMRNSPNTSPKLMRHDPLLIPGNDQIDNMDSNVKKYDSTGMFHWCAPKEIEKVILTRSEAAMTVLSGHVVVCIFGDVSSALIGLRNLVMPLRASNFHYHELKHIVFVGSIEYLKREWETLHNFPKVSILPGTPLSRADLRAVNINLCDMCVILSANQNNIDDTSLQDKECILASLNIKSMQFDDSIGVLQANSQGFTPPGMDRSSPDNSPVHGMLRQPSITTGVNIPIITELVNDTNVQFLDQDDDDDPDTELYLTQPFACGTAFAVSVLDSLMSATYFNDNILTLIRTLVTGGATPELEALIAEENALRGGYSTPQTLANRDRCRVAQLALLDGPFADLGDGGCYGDLFCKALKTYNMLCFGIYRLRDAHLSTPSQCTKRYVITNPPYEFELVPTDLIFCLMQFDHNAGQSRASLSHSSHSSQSSSKKSSSVHSIPSTANRQNRPKSRESRDKQKYVQEERL",
# KIF5A_HUMAN
"MAETNNECSIKVLCRFRPLNQAEILRGDKFIPIFQGDDSVVIGGKPYVFDRVFPPNTTQEQVYHACAMQIVKDVLAGYNGTIFAYGQTSSGKTHTMEGKLHDPQLMGIIPRIARDIFNHIYSMDENLEFHIKVSYFEIYLDKIRDLLDVTKTNLSVHEDKNRVPFVKGCTERFVSSPEEILDVIDEGKSNRHVAVTNMNEHSSRSHSIFLINIKQENMETEQKLSGKLYLVDLAGSEKVSKTGAEGAVLDEAKNINKSLSALGNVISALAEGTKSYVPYRDSKMTRILQDSLGGNCRTTMFICCSPSSYNDAETKSTLMFGQRAKTIKNTASVNLELTAEQWKKKYEKEKEKTKAQKETIAKLEAELSRWRNGENVPETERLAGEEAALGAELCEETPVNDNSSIVVRIAPEERQKYEEEIRRLYKQLDDKDDEINQQSQLIEKLKQQMLDQEELLVSTRGDNEKVQRELSHLQSENDAAKDEVKEVLQALEELAVNYDQKSQEVEEKSQQNQLLVDELSQKVATMLSLESELQRLQEVSGHQRKRIAEVLNGLMKDLSEFSVIVGNGEIKLPVEISGAIEEEFTVARLYISKIKSEVKSVVKRCRQLENLQVECHRKMEVTGRELSSCQLLISQHEAKIRSLTEYMQSVELKKRHLEESYDSLSDELAKLQAQETVHEVALKDKEPDTQDADEVKKALELQMESHREAHHRQLARLRDEINEKQKTIDELKDLNQKLQLELEKLQADYEKLKSEEHEKSTKLQELTFLYERHEQSKQDLKGLEETVARELQTLHNLRKLFVQDVTTRVKKSAEMEPEDSGGIHSQKQKISFLENNLEQLTKVHKQLVRDNADLRCELPKLEKRLRATAERVKALEGALKEAKEGAMKDKRRYQQEVDRIKEAVRYKSSGKRGHSAQIAKPVRPGHYPASSPTNPYGTRSPECISYTNSLFQNYQNLYLQATPSSTSDMYFANSCTSSGATSSGGPLASYQKANMDNGNATDINDNRSDLPCGYEAEDQAKLFPLHQETAAS",
# STX4_HUMAN
"MRDRTHELRQGDDSSDEEDKERVALVVHPGTARLGSPDEEFFHKVRTIRQTIVKLGNKVQELEKQQVTILATPLPEESMKQELQNLRDEIKQLGREIRLQLKAIEPQKEEADENYNSVNTRMRKTQHGVLSQQFVELINKCNSMQSEYREKNVERIRRQLKITNAGMVSDEELEQMLDSGQSEVFVSNILKDTQVTRQALNEISARHSEIQQLERSIRELHDIFTFLATEVEMQGEMINRIEKNILSSADYVERGQEHVKTALENQKKARKKKVLIAICVSITVVLLAVIIGVTVVG",
# CNTN1_HUMAN
"MKMWLLVSHLVIISITTCLAEFTWYRRYGHGVSEEDKGFGPIFEEQPINTIYPEESLEGKVSLNCRARASPFPVYKWRMNNGDVDLTSDRYSMVGGNLVINNPDKQKDAGIYYCLASNNYGMVRSTEATLSFGYLDPFPPEERPEVRVKEGKGMVLLCDPPYHFPDDLSYRWLLNEFPVFITMDKRRFVSQTNGNLYIANVEASDKGNYSCFVSSPSITKSVFSKFIPLIPIPERTTKPYPADIVVQFKDVYALMGQNVTLECFALGNPVPDIRWRKVLEPMPSTAEISTSGAVLKIFNIQLEDEGIYECEAENIRGKDKHQARIYVQAFPEWVEHINDTEVDIGSDLYWPCVATGKPIPTIRWLKNGYAYHKGELRLYDVTFENAGMYQCIAENTYGAIYANAELKILALAPTFEMNPMKKKILAAKGGRVIIECKPKAAPKPKFSWSKGTEWLVNSSRILIWEDGSLEINNITRNDGGIYTCFAENNRGKANSTGTLVITDPTRIILAPINADITVGENATMQCAASFDPALDLTFVWSFNGYVIDFNKENIHYQRNFMLDSNGELLIRNAQLKHAGRYTCTAQTIVDNSSASADLVVRGPPGPPGGLRIEDIRATSVALTWSRGSDNHSPISKYTIQTKTILSDDWKDAKTDPPIIEGNMEAARAVDLIPWMEYEFRVVATNTLGRGEPSIPSNRIKTDGAAPNVAPSDVGGGGGRNRELTITWAPLSREYHYGNNFGYIVAFKPFDGEEWKKVTVTNPDTGRYVHKDETMSPSTAFQVKVKAFNNKGDGPYSLVAVINSAQDAPSEAPTEVGVKVLSSSEISVHWEHVLEKIVESYQIRYWAAHDKEEAANRVQVTSQEYSARLENLLPDTQYFIEVGACNSAGCGPPSDMIEAFTKKAPPSQPPRIISSVRSGSRYIITWDHVVALSNESTVTGYKVLYRPDGQHDGKLYSTHKHSIEVPIPRDGEYVVEVRAHSDGGDGVVSQVKISGAPTLSPSLLGLLLPAFGILVYLEF",
# NMDE1_HUMAN
"MGRVGYWTLLVLPALLVWRGPAPSAAAEKGPPALNIAVMLGHSHDVTERELRTLWGPEQAAGLPLDVNVVALLMNRTDPKSLITHVCDLMSGARIHGLVFGDDTDQEAVAQMLDFISSHTFVPILGIHGGASMIMADKDPTSTFFQFGASIQQQATVMLKIMQDYDWHVFSLVTTIFPGYREFISFVKTTVDNSFVGWDMQNVITLDTSFEDAKTQVQLKKIHSSVILLYCSKDEAVLILSEARSLGLTGYDFFWIVPSLVSGNTELIPKEFPSGLISVSYDDWDYSLEARVRDGIGILTTAASSMLEKFSYIPEAKASCYGQMERPEVPMHTLHPFMVNVTWDGKDLSFTEEGYQVHPRLVVIVLNKDREWEKVGKWENHTLSLRHAVWPRYKSFSDCEPDDNHLSIVTLEEAPFVIVEDIDPLTETCVRNTVPCRKFVKINNSTNEGMNVKKCCKGFCIDILKKLSRTVKFTYDLYLVTNGKHGKKVNNVWNGMIGEVVYQRAVMAVGSLTINEERSEVVDFSVPFVETGISVMVSRSNGTVSPSAFLEPFSASVWVMMFVMLLIVSAIAVFVFEYFSPVGYNRNLAKGKAPHGPSFTIGKAIWLLWGLVFNNSVPVQNPKGTTSKIMVSVWAFFAVIFLASYTANLAAFMIQEEFVDQVTGLSDKKFQRPHDYSPPFRFGTVPNGSTERNIRNNYPYMHQYMTKFNQKGVEDALVSLKTGKLDAFIYDAAVLNYKAGRDEGCKLVTIGSGYIFATTGYGIALQKGSPWKRQIDLALLQFVGDGEMEELETLWLTGICHNEKNEVMSSQLDIDNMAGVFYMLAAAMALSLITFIWEHLFYWKLRFCFTGVCSDRPGLLFSISRGIYSCIHGVHIEEKKKSPDFNLTGSQSNMLKLLRSAKNISSMSNMNSSRMDSPKRAADFIQRGSLIMDMVSDKGNLMYSDNRSFQGKESIFGDNMNELQTFVANRQKDNLNNYVFQGQHPLTLNESNPNTVEVAVSTESKANSRPRQLWKKSVDSIRQDSLSQNPVSQRDEATAENRTHSLKSPRYLPEEMAHSDISETSNRATCHREPDNSKNHKTKDNFKRSVASKYPKDCSEVERTYLKTKSSSPRDKIYTIDGEKEPGFHLDPPQFVENVTLPENVDFPDPYQDPSENFRKGDSTLPMNRNPLHNEEGLSNNDQYKLYSKHFTLKDKGSPHSETSERYRQNSTHCRSCLSNMPTYSGHFTMRSPFKCDACLRMGNLYDIDEDQMLQETGNPATGEQVYQQDWAQNNALQLQKNKLRISRQHSYDNIVDKPRELDLSRPSRSISLKDRERLLEGNFYGSLFSVPSSKLSGKKSSLFPQGLEDSKRSKSLLPDHTSDNPFLHSHRDDQRLVIGRCPSDPYKHSLPSQAVNDSYLRSSLRSTASYCSRDSRGHNDVYISEHVMPYAANKNNMYSTPRVLNSCSNRRVYKKMPSIESDV",
# ELAV2_HUMAN
"METQLSNGPTCNNTANGPTTINNNCSSPVDSGNTEDSKTNLIVNYLPQNMTQEELKSLFGSIGEIESCKLVRDKITGQSLGYGFVNYIDPKDAEKAINTLNGLRLQTKTIKVSYARPSSASIRDANLYVSGLPKTMTQKELEQLFSQYGRIITSRILVDQVTGISRGVGFIRFDKRIEAEEAIKGLNGQKPPGATEPITVKFANNPSQKTNQAILSQLYQSPNRRYPGPLAQQAQRFRLDNLLNMAYGVKRFSPMTIDGMTSLAGINIPGHPGTGWCIFVYNLAPDADESILWQMFGPFGAVTNVKVIRDFNTNKCKGFGFVTMTNYDEAAMAIASLNGYRLGDRVLQVSFKTNKTHKA",
# EPS8_HUMAN
"MNGHISNHPSSFGMYPSQMNGYGSSPTFSQTDREHGSKTSAKALYEQRKNYARDSVSSVSDISQYRVEHLTTFVLDRKDAMITVDDGIRKLKLLDAKGKVWTQDMILQVDDRAVSLIDLESKNELENFPLNTIQHCQAVMHSCSYDSVLALVCKEPTQNKPDLHLFQCDEVKANLISEDIESAISDSKGGKQKRRPDALRMISNADPSIPPPPRAPAPAPPGTVTQVDVRSRVAAWSAWAADQGDFEKPRQYHEQEETPEMMAARIDRDVQILNHILDDIEFFITKLQKAAEAFSELSKRKKNKKGKRKGPGEGVLTLRAKPPPPDEFLDCFQKFKHGFNLLAKLKSHIQNPSAADLVHFLFTPLNMVVQATGGPELASSVLSPLLNKDTIDFLNYTVNGDERQLWMSLGGTWMKARAEWPKEQFIPPYVPRFRNGWEPPMLNFMGATMEQDLYQLAESVANVAEHQRKQEIKRLSTEHSSVSEYHPADGYAFSSNIYTRGSHLDQGEAAVAFKPTSNRHIDRNYEPLKTQPKKYAKSKYDFVARNNSELSVLKDDILEILDDRKQWWKVRNASGDSGFVPNNILDIVRPPESGLGRADPPYTHTIQKQRMEYGPRPADTPPAPSPPPTPAPVPVPLPPSTPAPVPVSKVPANITRQNSSSSDSGGSIVRDSQRHKQLPVDRRKSQMEEVQDELIHRLTIGRSAAQKKFHVPRQNVPVINITYDSTPEDVKTWLQSKGFNPVTVNSLGVLNGAQLFSLNKDELRTVCPEGARVYSQITVQKAALEDSSGSSELQEIMRRRQEKISAAASDSGVESFDEGSSH",
# ANK3_HUMAN
"MAHAASQLKKNRDLEINAEEEPEKKRKHRKRSRDRKKKSDANASYLRAARAGHLEKALDYIKNGVDINICNQNGLNALHLASKEGHVEVVSELLQREANVDAATKKGNTALHIASLAGQAEVVKVLVTNGANVNAQSQNGFTPLYMAAQENHLEVVKFLLDNGASQSLATEDGFTPLAVALQQGHDQVVSLLLENDTKGKVRLPALHIAARKDDTKAAALLLQNDNNADVESKSGFTPLHIAAHYGNINVATLLLNRAAAVDFTARNDITPLHVASKRGNANMVKLLLDRGAKIDAKTRDGLTPLHCGARSGHEQVVEMLLDRAAPILSKTKNGLSPLHMATQGDHLNCVQLLLQHNVPVDDVTNDYLTALHVAAHCGHYKVAKVLLDKKANPNAKALNGFTPLHIACKKNRIKVMELLLKHGASIQAVTESGLTPIHVAAFMGHVNIVSQLMHHGASPNTTNVRGETALHMAARSGQAEVVRYLVQDGAQVEAKAKDDQTPLHISARLGKADIVQQLLQQGASPNAATTSGYTPLHLSAREGHEDVAAFLLDHGASLSITTKKGFTPLHVAAKYGKLEVANLLLQKSASPDAAGKSGLTPLHVAAHYDNQKVALLLLDQGASPHAAAKNGYTPLHIAAKKNQMDIATTLLEYGADANAVTRQGIASVHLAAQEGHVDMVSLLLGRNANVNLSNKSGLTPLHLAAQEDRVNVAEVLVNQGAHVDAQTKMGYTPLHVGCHYGNIKIVNFLLQHSAKVNAKTKNGYTPLHQAAQQGHTHIINVLLQNNASPNELTVNGNTALGIARRLGYISVVDTLKIVTEETMTTTTVTEKHKMNVPETMNEVLDMSDDEVRKANAPEMLSDGEYISDVEEGEDAMTGDTDKYLGPQDLKELGDDSLPAEGYMGFSLGARSASLRSFSSDRSYTLNRSSYARDSMMIEELLVPSKEQHLTFTREFDSDSLRHYSWAADTLDNVNLVSSPIHSGFLVSFMVDARGGSMRGSRHHGMRIIIPPRKCTAPTRITCRLVKRHKLANPPPMVEGEGLASRLVEMGPAGAQFLGPVIVEIPHFGSMRGKERELIVLRSENGETWKEHQFDSKNEDLTELLNGMDEELDSPEELGKKRICRIITKDFPQYFAVVSRIKQESNQIGPEGGILSSTTVPLVQASFPEGALTKRIRVGLQAQPVPDEIVKKILGNKATFSPIVTVEPRRRKFHKPITMTIPVPPPSGEGVSNGYKGDTTPNLRLLCSITGGTSPAQWEDITGTTPLTFIKDCVSFTTNVSARFWLADCHQVLETVGLATQLYRELICVPYMAKFVVFAKMNDPVESSLRCFCMTDDKVDKTLEQQENFEEVARSKDIEVLEGKPIYVDCYGNLAPLTKGGQQLVFNFYSFKENRLPFSIKIRDTSQEPCGRLSFLKEPKTTKGLPQTAVCNLNITLPAHKKETESDQDDEIEKTDRRQSFASLALRKRYSYLTEPGMIERSTGATRSLPTTYSYKPFFSTRPYQSWTTAPITVPGPAKSGFTSLSSSSSNTPSASPLKSIWSVSTPSPIKSTLGASTTSSVKSISDVASPIRSFRTMSSPIKTVVSQSPYNIQVSSGTLARAPAVTEATPLKGLASNSTFSSRTSPVTTAGSLLERSSITMTPPASPKSNINMYSSSLPFKSIITSAAPLISSPLKSVVSPVKSAVDVISSAKITMASSLSSPVKQMPGHAEVALVNGSISPLKYPSSSTLINGCKATATLQEKISSATNSVSSVVSAATDTVEKVFSTTTAMPFSPLRSYVSAAPSAFQSLRTPSASALYTSLGSSISATTSSVTSSIITVPVYSVVNVLPEPALKKLPDSNSFTKSAAALLSPIKTLTTETHPQPHFSRTSSPVKSSLFLAPSALKLSTPSSLSSSQEILKDVAEMKEDLMRMTAILQTDVPEEKPFQPELPKEGRIDDEEPFKIVEKVKEDLVKVSEILKKDVCVDNKGSPKSPKSDKGHSPEDDWIEFSSEEIREARQQAAASQSPSLPERVQVKAKAASEKDYNLTKVIDYLTNDIGSSSLTNLKYKFEDAKKDGEERQKRVLKPAIALQEHKLKMPPASMRTSTSEKELCKMADSFFGTDTILESPDDFSQHDQDKSPLSDSGFETRSEKTPSAPQSAESTGPKPLFHEVPIPPVITETRTEVVHVIRSYDPSAGDVPQTQPEEPVSPKPSPTFMELEPKPTTSSIKEKVKAFQMKASSEEDDHNRVLSKGMRVKEETHITTTTRMVYHSPPGGEGASERIEETMSVHDIMKAFQSGRDPSKELAGLFEHKSAVSPDVHKSAAETSAQHAEKDNQMKPKLERIIEVHIEKGNQAEPTEVIIRETKKHPEKEMYVYQKDLSRGDINLKDFLPEKHDAFPCSEEQGQQEEEELTAEESLPSYLESSRVNTPVSQEEDSRPSSAQLISDDSYKTLKLLSQHSIEYHDDELSELRGESYRFAEKMLLSEKLDVSHSDTEESVTDHAGPPSSELQGSDKRSREKIATAPKKEILSKIYKDVSENGVGKVSKDEHFDKVTVLHYSGNVSSPKHAMWMRFTEDRLDRGREKLIYEDRVDRTVKEAEEKLTEVSQFFRDKTEKLNDELQSPEKKARPKNGKEYSSQSPTSSSPEKVLLTELLASNDEWVKARQHGPDGQGFPKAEEKAPSLPSSPEKMVLSQQTEDSKSTVEAKGSISQSKAPDGPQSGFQLKQSKLSSIRLKFEQGTHAKSKDMSQEDRKSDGQSRIPVKKIQESKLPVYQVFAREKQQKAIDLPDESVSVQKDFMVLKTKDEHAQSNEIVVNDSGSDNVKKQRTEMSSKAMPDSFSEQQAKDLACHITSDLATRGPWDKKVFRTWESSGATNNKSQKEKLSHVLVHDVRENHIGHPESKSVDQKNEFMSVTERERKLLTNGSLSEIKEMTVKSPSKKVLYREYVVKEGDHPGGLLDQPSRRSESSAVSHIPVRVADERRMLSSNIPDGFCEQSAFPKHELSQKLSQSSMSKETVETQHFNSIEDEKVTYSEISKVSKHQSYVGLCPPLEETETSPTKSPDSLEFSPGKESPSSDVFDHSPIDGLEKLAPLAQTEGGKEIKTLPVYVSFVQVGKQYEKEIQQGGVKKIISQECKTVQETRGTFYTTRQQKQPPSPQGSPEDDTLEQVSFLDSSGKSPLTPETPSSEEVSYEFTSKTPDSLIAYIPGKPSPIPEVSEESEEEEQAKSTSLKQTTVEETAVEREMPNDVSKDSNQRPKNNRVAYIEFPPPPPLDADQIESDKKHHYLPEKEVDMIEVNLQDEHDKYQLAEPVIRVQPPSPVPPGADVSDSSDDESIYQPVPVKKYTFKLKEVDDEQKEKPKASAEKASNQKELESNGSGKDNEFGLGLDSPQNEIAQNGNNDQSITECSIATTAEFSHDTDATEIDSLDGYDLQDEDDGLTESDSKLPIQAMEIKKDIWNTEGILKPADRSFSQSKLEVIEEEGKVGPDEDKPPSKSSSSEKTPDKTDQKSGAQFFTLEGRHPDRSVFPDTYFSYKVDEEFATPFKTVATKGLDFDPWSNNRGDDEVFDSKSREDETKPFGLAVEDRSPATTPDTTPARTPTDESTPTSEPNPFPFHEGKMFEMTRSGAIDMSKRDFVEERLQFFQIGEHTSEGKSGDQGEGDKSMVTATPQPQSGDTTVETNLERNVETPTVEPNPSIPTSGECQEGTSSSGSLEKSAAATNTSKVDPKLRTPIKMGISASTMTMKKEGPGEITDKIEAVMTSCQGLENETITMISNTANSQMGVRPHEKHDFQKDNFNNNNNLDSSTIQTDNIMSNIVLTEHSAPTCTTEKDNPVKVSSGKKTGVLQGHCVRDKQKVLGEQQKTKELIGIRQKSKLPIKATSPKDTFPPNHMSNTKASKMKQVSQSEKTKALTTSSCVDVKSRIPVKNTHRDNIIAVRKACATQKQGQPEKGKAKQLPSKLPVKVRSTCVTTTTTTATTTTTTTTTTTTSCTVKVRKSQLKEVCKHSIEYFKGISGETLKLVDRLSEEEKKMQSELSDEEESTSRNTSLSETSRGGQPSVTTKSARDKKTEAAPLKSKSEKAGSEKRSSRRTGPQSPCERTDIRMAIVADHLGLSWTELARELNFSVDEINQIRVENPNSLISQSFMLLKKWVTRDGKNATTDALTSVLTKINRIDIVTLLEGPIFDYGNISGTRSFADENNVFHDPVDGWQNETSSGNLESCAQARRVTGGLLDRLDDSPDQCRDSITSYLKGEAGKFEANGSHTEITPEAKTKSYFPESQNDVGKQSTKETLKPKIHGSGHVEEPASPLAAYQKSLEETSKLIIEETKPCVPVSMKKMSRTSPADGKPRLSLHEEEGSSGSEQKQGEGFKVKTKKEIRHVEKKSHS",
# DLG1_HUMAN
"MPVRKQDTQRALHLLEEYRSKLSQTEDRQLRSSIERVINIFQSNLFQALIDIQEFYEVTLLDNPKCIDRSKPSEPIQPVNTWEISSLPSSTVTSETLPSSLSPSVEKYRYQDEDTPPQEHISPQITNEVIGPELVHVSEKNLSEIENVHGFVSHSHISPIKPTEAVLPSPPTVPVIPVLPVPAENTVILPTIPQANPPPVLVNTDSLETPTYVNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRLRVNDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTALKNTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSPSSFLGQTPASPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKIHDLREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVINASDDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKTRDKGEIPDDMGSKGLKHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPDKFGSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVREVAEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERAMKLEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL",
# ABR_HUMAN
"MEPLSHRGLPRLSWIDTLYSNFSYGTDEYDGEGNEEQKGPPEGSETMPYIDESPTMSPQLSARSQGGGDGVSPTPPEGLAPGVEAGKGLEMRKLVLSGFLASEEIYINQLEALLLPMKPLKATATTSQPVLTIQQIETIFYKIQDIYEIHKEFYDNLCPKVQQWDSQVTMGHLFQKLASQLGVYKAFVDNYKVALETAEKCSQSNNQFQKISEELKVKGPKDSKDSHTSVTMEALLYKPIDRVTRSTLVLHDLLKHTPVDHPDYPLLQDALRISQNFLSSINEDIDPRRTAVTTPKGETRQLVKDGFLVEVSESSRKLRHVFLFTDVLLCAKLKKTSAGKHQQYDCKWYIPLADLVFPSPEESEASPQVHPFPDHELEDMKMKISALKSEIQKEKANKGQSRAIERLKKKMFENEFLLLLNSPTIPFRIHNRNGKSYLFLLSSDYERSEWREAIQKLQKKDLQAFVLSSVELQVLTGSCFKLRTVHNIPVTSNKDDDESPGLYGFLHVIVHSAKGFKQSANLYCTLEVDSFGYFVSKAKTRVFRDTAEPKWDEEFEIELEGSQSLRILCYEKCYDKTKVNKDNNEIVDKIMGKGQIQLDPQTVETKNWHTDVIEMNGIKVEFSMKFTSRDMSLKRTPSKKQTGVFGVKISVVTKRERSKVPYIVRQCVEEVEKRGIEEVGIYRISGVATDIQALKAVFDANNKDILLMLSDMDINAIAGTLKLYFRELPEPLLTDRLYPAFMEGIALSDPAAKENCMMHLLRSLPDPNLITFLFLLEHLKRVAEKEPINKMSLHNLATVFGPTLLRPSEVESKAHLTSAADIWSHDVMAQVQVLLYYLQHPPISFAELKRNTLYFSTDV",
# GRIK2_HUMAN
"MKIIFPILSNPVFRRTVKLLLCLLWIGYSQGTTHVLRFGGIFEYVESGPMGAEELAFRFAVNTINRNRTLLPNTTLTYDTQKINLYDSFEASKKACDQLSLGVAAIFGPSHSSSANAVQSICNALGVPHIQTRWKHQVSDNKDSFYVSLYPDFSSLSRAILDLVQFFKWKTVTVVYDDSTGLIRLQELIKAPSRYNLRLKIRQLPADTKDAKPLLKEMKRGKEFHVIFDCSHEMAAGILKQALAMGMMTEYYHYIFTTLDLFALDVEPYRYSGVNMTGFRILNTENTQVSSIIEKWSMERLQAPPKPDSGLLDGFMTTDAALMYDAVHVVSVAVQQFPQMTVSSLQCNRHKPWRFGTRFMSLIKEAHWEGLTGRITFNKTNGLRTDFDLDVISLKEEGLEKIGTWDPASGLNMTESQKGKPANITDSLSNRSLIVTTILEEPYVLFKKSDKPLYGNDRFEGYCIDLLRELSTILGFTYEIRLVEDGKYGAQDDANGQWNGMVRELIDHKADLAVAPLAITYVREKVIDFSKPFMTLGISILYRKPNGTNPGVFSFLNPLSPDIWMYILLAYLGVSCVLFVIARFSPYEWYNPHPCNPDSDVVENNFTLLNSFWFGVGALMQQGSELMPKALSTRIVGGIWWFFTLIIISSYTANLAAFLTVERMESPIDSADDLAKQTKIEYGAVEDGATMTFFKKSKISTYDKMWAFMSSRRQSVLVKSNEEGIQRVLTSDYAFLMESTTIEFVTQRNCNLTQIGGLIDSKGYGVGTPMGSPYRDKITIAILQLQEEGKLHMMKEKWWRGNGCPEEESKEASALGVQNIGGIFIVLAAGLVLSVFVAVGEFLYKSKKNAQLEKRSFCSAMVEELRMSLKCQRRLKHKPQAPVIVKTEEVINMHTFNDRRLPGKETMA",
# GRIK3_HUMAN
"MTAPWRRLRSLVWEYWAGLLVCAFWIPDSRGMPHVIRIGGIFEYADGPNAQVMNAEEHAFRFSANIINRNRTLLPNTTLTYDIQRIHFHDSFEATKKACDQLALGVVAIFGPSQGSCTNAVQSICNALEVPHIQLRWKHHPLDNKDTFYVNLYPDYASLSHAILDLVQYLKWRSATVVYDDSTGLIRLQELIMAPSRYNIRLKIRQLPIDSDDSRPLLKEMKRGREFRIIFDCSHTMAAQILKQAMAMGMMTEYYHFIFTTLDLYALDLEPYRYSGVNLTGFRILNVDNPHVSAIVEKWSMERLQAAPRSESGLLDGVMMTDAALLYDAVHIVSVCYQRAPQMTVNSLQCHRHKAWRFGGRFMNFIKEAQWEGLTGRIVFNKTSGLRTDFDLDIISLKEDGLEKVGVWSPADGLNITEVAKGRGPNVTDSLTNRSLIVTTVLEEPFVMFRKSDRTLYGNDRFEGYCIDLLKELAHILGFSYEIRLVEDGKYGAQDDKGQWNGMVKELIDHKADLAVAPLTITHVREKAIDFSKPFMTLGVSILYRKPNGTNPSVFSFLNPLSPDIWMYVLLAYLGVSCVLFVIARFSPYEWYDAHPCNPGSEVVENNFTLLNSFWFGMGSLMQQGSELMPKALSTRIIGGIWWFFTLIIISSYTANLAAFLTVERMESPIDSADDLAKQTKIEYGAVKDGATMTFFKKSKISTFEKMWAFMSSKPSALVKNNEEGIQRALTADYALLMESTTIEYVTQRNCNLTQIGGLIDSKGYGIGTPMGSPYRDKITIAILQLQEEDKLHIMKEKWWRGSGCPEEENKEASALGIQKIGGIFIVLAAGLVLSVLVAVGEFVYKLRKTAEREQRSFCSTVADEIRFSLTCQRRVKHKPQPPMMVKTDAVINMHTFNDRRLPGKDSMACSTSLAPVFP",
# TIAM1_HUMAN
"MGNAESQHVEHEFYGEKHASLGRKHTSRSLRLSHKTRRTRHASSGKVIHRNSEVSTRSSSTPSIPQSLAENGLEPFSQDGTLEDFGSPIWVDRVDMGLRPVSYTDSSVTPSVDSSIVLTAASVQSMPDTEESRLYGDDATYLAEGGRRQHSYTSNGPTFMETASFKKKRSKSADIWREDSLEFSLSDLSQEHLTSNEEILGSAEEKDCEEARGMETRASPRQLSTCQRANSLGDLYAQKNSGVTANGGPGSKFAGYCRNLVSDIPNLANHKMPPAAAEETPPYSNYNTLPCRKSHCLSEGATNPQISHSNSMQGRRAKTTQDVNAGEGSEFADSGIEGATTDTDLLSRRSNATNSSYSPTTGRAFVGSDSGSSSTGDAARQGVYENFRRELEMSTTNSESLEEAGSAHSDEQSSGTLSSPGQSDILLTAAQGTVRKAGALAVKNFLVHKKNKKVESATRRKWKHYWVSLKGCTLFFYESDGRSGIDHNSIPKHAVWVENSIVQAVPEHPKKDFVFCLSNSLGDAFLFQTTSQTELENWITAIHSACATAVARHHHKEDTLRLLKSEIKKLEQKIDMDEKMKKMGEMQLSSVTDSKKKKTILDQIFVWEQNLEQFQMDLFRFRCYLASLQGGELPNPKRLLAFASRPTKVAMGRLGIFSVSSFHALVAARTGETGVRRRTQAMSRSASKRRSRFSSLWGLDTTSKKKQGRPSINQVFGEGTEAVKKSLEGIFDDIVPDGKREKEVVLPNVHQHNPDCDIWVHEYFTPSWFCLPNNQPALTVVRPGDTARDTLELICKTHQLDHSAHYLRLKFLIENKMQLYVPQPEEDIYELLYKEIEICPKVTQSIHIEKSDTAADTYGFSLSSVEEDGIRRLYVNSVKETGLASKKGLKAGDEILEINNRAADALNSSMLKDFLSQPSLGLLVRTYPELEEGVELLESPPHRVDGPADLGESPLAFLTSNPGHSLCSEQGSSAETAPEETEGPDLESSDETDHSSKSTEQVAAFCRSLHEMNPSDQSPSPQDSTGPQLATMRQLSDADKLRKVICELLETERTYVKDLNCLMERYLKPLQKETFLTQDELDVLFGNLTEMVEFQVEFLKTLEDGVRLVPDLEKLEKVDQFKKVLFSLGGSFLYYADRFKLYSAFCASHTKVPKVLVKAKTDTAFKAFLDAQNPKQQHSSTLESYLIKPIQRILKYPLLLRELFALTDAESEEHYHLDVAIKTMNKVASHINEMQKIHEEFGAVFDQLIAEQTGEKKEVADLSMGDLLLHTTVIWLNPPASLGKWKKEPELAAFVFKTAVVLVYKDGSKQKKKLVGSHRLSIYEDWDPFRFRHMIPTEALQVRALASADAEANAVCEIVHVKSESEGRPERVFHLCCSSPESRKDFLKAVHSILRDKHRRQLLKTESLPSSQQYVPFGGKRLCALKGARPAMSRAVSAPSKSLGRRRRRLARNRFTIDSDAVSASSPEKESQQPPGGGDTDRWVEEQFDLAQYEEQDDIKETDILSDDDEFCESVKGASVDRDLQERLQATSISQRERGRKTLDSHASRMAQLKKQAALSGINGGLESASEEVIWVRREDFAPSRKLNTEI",
# STRN3_HUMAN
"MDELAGGGGGGPGMAAPPRQQQGPGGNLGLSPGGNGAAGGGGPPASEGAGPAAGPELSRPQQYTIPGILHYIQHEWARFEMERAHWEVERAELQARIAFLQGERKGQENLKKDLVRRIKMLEYALKQERAKYHKLKYGTELNQGDLKMPTFESEETKDTEAPTAPQNSQLTWKQGRQLLRQYLQEVGYTDTILDVRSQRVRSLLGLSNSEPNGSVETKNLEQILNGGESPKQKGQEIKRSSGDVLETFNFLENADDSDEDEENDMIEGIPEGKDKHRMNKHKIGNEGLAADLTDDPDTEEALKEFDFLVTAEDGEGAGEARSSGDGTEWDKDDLSPTAEVWDVDQGLISKLKEQYKKERKGKKGVKRANRTKLYDMIADLGDDELPHIPSGIINQSRSASTRMTDHEGARAEEAEPITFPSGGGKSFIMGSDDVLLSVLGLGDLADLTVTNDADYSYDLPANKDAFRKTWNPKYTLRSHFDGVRALAFHPVEPVLVTASEDHTLKLWNLQKTVPAKKSASLDVEPIYTFRAHIGPVLSLAISSNGEQCFSGGIDATIQWWNMPSPSVDPYDTYEPNVLAGTLVGHTDAVWGLAYSGIKNQLLSCSADGTVRLWNPQEKLPCICTYNGDKKHGIPTSVDFIGCDPAHMVTSFNTGSAVIYDLETSQSLVILSSQVDSGLQSNNHINRVVSHPTLPVTITAHEDRHIKFFDNKTGKMIHSMVAHLDAVTSLAVDPNGIYLMSGSHDCSIRLWNLDSKTCVQEITAHRKKLDESIYDVAFHSSKAYIASAGADALAKVFV",
# LIPA1_HUMAN
"MMCEVMPTISEAEGPPGGGGGHGSGSPSQPDADSHFEQLMVSMLEERDRLLDTLRETQETLALTQGKLHEVGHERDSLQRQLNTALPQEFAALTKELNVCREQLLEREEEIAELKAERNNTRLLLEHLECLVSRHERSLRMTVVKRQAQSPAGVSSEVEVLKALKSLFEHHKALDEKVRERLRVALERCSLLEEELGATHKELMILKEQNNQKKTLTDGVLDINHEQENTPSTSGKRSSDGSLSHEEDLAKVIELQEIISKQSREQSQMKERLASLSSHVTELEEDLDTARKDLIKSEEMNTKLQRDVREAMAQKEDMEERITTLEKRYLAAQREATSVHDLNDKLENEIANKDSMHRQTEDKNRQLQERLELAEQKLQQTLRKAETLPEVEAELAQRVAALSKAEERHGNIEERLRQMEAQLEEKNQELQRARQREKMNEEHNKRLSDTVDKLLSESNERLQLHLKERMAALEDKNSLLREVESAKKQLEETQHDKDQLVLNIEALRAELDHMRLRGASLHHGRPHLGSVPDFRFPMADGHTDSYSTSAVLRRPQKGRLAALRDEPSKVQTLNEQDWERAQQASVLANVAQAFESDADVSDGEDDRDTLLSSVDLLSPSGQADAHTLAMMLQEQLDAINKEIRLIQEEKENTEQRAEEIESRVGSGSLDNLGRFRSMSSIPPYPASSLASSSPPGSGRSTPRRIPHSPAREVDRLGVMTLLPPSREEVRDDKTTIKCETSPPSSPRALRLDRLHKGALHTVSHEDIRDIRNSTGSQDGPVSNPSSSNSSQDSLHKAPKKKGIKSSIGRLFGKKEKGRPGQTGKEALGQAGVSETDNSSQDALGLSKLGGQAEKNRKLQKKHELLEEARRQGLPFAQWDGPTVVVWLELWVGMPAWYVAACRANVKSGAIMSALSDTEIQREIGISNPLHRLKLRLAIQEIMSLTSPSAPPTSRTTLAYGDMNHEWIGNEWLPSLGLPQYRSYFMECLVDARMLDHLTKKDLRGQLKMVDSFHRNSFQCGIMCLRRLNYDRKELERKREESQSEIKDVLVWSNDRVIRWILSIGLKEYANNLIESGVHGALLALDETFDFSALALLLQIPTQNTQARAVLEREFNNLLVMGTDRRFDEDDDKSFRRAPSWRKKFRPKDIRGLAAGSAETLPANFRVTSSMSSPSMQPKKMQMDGNVSGTQRLDSATVRTYSC",
# ROA0_HUMAN
"MENSQLCKLFIGGLNVQTSESGLRGHFEAFGTLTDCVVVVNPQTKRSRCFGFVTYSNVEEADAAMAASPHAVDGNTVELKRAVSREDSARPGAHAKVKKLFVGGLKGDVAEGDLIEHFSQFGTVEKAEIIADKQSGKKRGFGFVYFQNHDAADKAAVVKFHPIQGHRVEVKKAVPKEDIYSGGGGGGSRSSRGGRGGRGRGGGRDQNGLSKGGGGGYNSYGGYGGGGGGGYNAYGGGGGGSSYGGSDYGNGFGGFGSYSQHQSSYGPMKSGGGGGGGGSSWGGRSNSGPYRGGYGGGGGYGGSSF",
# PAK1_HUMAN
"MSNNGLDIQDKPPAPPMRNTSTMIGAGSKDAGTLNHGSKPLPPNPEEKKKKDRFYRSILPGDKTNKKKEKERPEISLPSDFEHTIHVGFDAVTGEFTGMPEQWARLLQTSNITKSEQKKNPQAVLDVLEFYNSKKTSNSQKYMSFTDKSAEDYNSSNALNVKAVSETPAVPPVSEDEDDDDDDATPPPVIAPRPEHTKSVYTRSVIEPLPVTPTRDVATSPISPTENNTTPPDALTRNTEKQKKKPKMSDEEILEKLRSIVSVGDPKKKYTRFEKIGQGASGTVYTAMDVATGQEVAIKQMNLQQQPKKELIINEILVMRENKNPNIVNYLDSYLVGDELWVVMEYLAGGSLTDVVTETCMDEGQIAAVCRECLQALEFLHSNQVIHRDIKSDNILLGMDGSVKLTDFGFCAQITPEQSKRSTMVGTPYWMAPEVVTRKAYGPKVDIWSLGIMAIEMIEGEPPYLNENPLRALYLIATNGTPELQNPEKLSAIFRDFLNRCLEMDVEKRGSAKELLQHQFLKIAKPLSSLTPLIAAAKEATKNNH",
# PAK2_HUMAN
"MSDNGELEDKPPAPPVRMSSTIFSTGGKDPLSANHSLKPLPSVPEEKKPRHKIISIFSGTEKGSKKKEKERPEISPPSDFEHTIHVGFDAVTGEFTGMPEQWARLLQTSNITKLEQKKNPQAVLDVLKFYDSNTVKQKYLSFTPPEKDGFPSGTPALNAKGTEAPAVVTEEEDDDEETAPPVIAPRPDHTKSIYTRSVIDPVPAPVGDSHVDGAAKSLDKQKKKTKMTDEEIMEKLRTIVSIGDPKKKYTRYEKIGQGASGTVFTATDVALGQEVAIKQINLQKQPKKELIINEILVMKELKNPNIVNFLDSYLVGDELFVVMEYLAGGSLTDVVTETCMDEAQIAAVCRECLQALEFLHANQVIHRDIKSDNVLLGMEGSVKLTDFGFCAQITPEQSKRSTMVGTPYWMAPEVVTRKAYGPKVDIWSLGIMAIEMVEGEPPYLNENPLRALYLIATNGTPELQNPEKLSPIFRDFLNRCLEMDVEKRGSAKELLQHPFLKLAKPLSSLTPLIMAAKEAMKSNR",
# CBLB_HUMAN
"MANSMNGRNPGGRGGNPRKGRILGIIDAIQDAVGPPKQAAADRRTVEKTWKLMDKVVRLCQNPKLQLKNSPPYILDILPDTYQHLRLILSKYDDNQKLAQLSENEYFKIYIDSLMKKSKRAIRLFKEGKERMYEEQSQDRRNLTKLSLIFSHMLAEIKAIFPNGQFQGDNFRITKADAAEFWRKFFGDKTIVPWKVFRQCLHEVHQISSGLEAMALKSTIDLTCNDYISVFEFDIFTRLFQPWGSILRNWNFLAVTHPGYMAFLTYDEVKARLQKYSTKPGSYIFRLSCTRLGQWAIGYVTGDGNILQTIPHNKPLFQALIDGSREGFYLYPDGRSYNPDLTGLCEPTPHDHIKVTQEQYELYCEMGSTFQLCKICAENDKDVKIEPCGHLMCTSCLTAWQESDGQGCPFCRCEIKGTEPIIVDPFDPRDEGSRCCSIIDPFGMPMLDLDDDDDREESLMMNRLANVRKCTDRQNSPVTSPGSSPLAQRRKPQPDPLQIPHLSLPPVPPRLDLIQKGIVRSPCGSPTGSPKSSPCMVRKQDKPLPAPPPPLRDPPPPPPERPPPIPPDNRLSRHIHHVESVPSRDPPMPLEAWCPRDVFGTNQLVGCRLLGEGSPKPGITASSNVNGRHSRVGSDPVLMRKHRRHDLPLEGAKVFSNGHLGSEEYDVPPRLSPPPPVTTLLPSIKCTGPLANSLSEKTRDPVEEDDDEYKIPSSHPVSLNSQPSHCHNVKPPVRSCDNGHCMLNGTHGPSSEKKSNIPDLSIYLKGDVFDSASDPVPLPPARPPTRDNPKHGSSLNRTPSDYDLLIPPLGEDAFDALPPSLPPPPPPARHSLIEHSKPPGSSSRPSSGQDLFLLPSDPFVDLASGQVPLPPARRLPGENVKTNRTSQDYDQLPSCSDGSQAPARPPKPRPRRTAPEIHHRKPHGPEAALENVDAKIAKLMGEGYAFEEVKRALEIAQNNVEVARSILREFAFPPPVSPRLNL",
# NMDE2_HUMAN
"MKPRAECCSPKFWLVLAVLAVSGSRARSQKSPPSIGIAVILVGTSDEVAIKDAHEKDDFHHLSVVPRVELVAMNETDPKSIITRICDLMSDRKIQGVVFADDTDQEAIAQILDFISAQTLTPILGIHGGSSMIMADKDESSMFFQFGPSIEQQASVMLNIMEEYDWYIFSIVTTYFPGYQDFVNKIRSTIENSFVGWELEEVLLLDMSLDDGDSKIQNQLKKLQSPIILLYCTKEEATYIFEVANSVGLTGYGYTWIVPSLVAGDTDTVPAEFPTGLISVSYDEWDYGLPARVRDGIAIITTAASDMLSEHSFIPEPKSSCYNTHEKRIYQSNMLNRYLINVTFEGRNLSFSEDGYQMHPKLVIILLNKERKWERVGKWKDKSLQMKYYVWPRMCPETEEQEDDHLSIVTLEEAPFVIVESVDPLSGTCMRNTVPCQKRIVTENKTDEEPGYIKKCCKGFCIDILKKISKSVKFTYDLYLVTNGKHGKKINGTWNGMIGEVVMKRAYMAVGSLTINEERSEVVDFSVPFIETGISVMVSRSNGTVSPSAFLEPFSADVWVMMFVMLLIVSAVAVFVFEYFSPVGYNRCLADGREPGGPSFTIGKAIWLLWGLVFNNSVPVQNPKGTTSKIMVSVWAFFAVIFLASYTANLAAFMIQEEYVDQVSGLSDKKFQRPNDFSPPFRFGTVPNGSTERNIRNNYAEMHAYMGKFNQRGVDDALLSLKTGKLDAFIYDAAVLNYMAGRDEGCKLVTIGSGKVFASTGYGIAIQKDSGWKRQVDLAILQLFGDGEMEELEALWLTGICHNEKNEVMSSQLDIDNMAGVFYMLGAAMALSLITFICEHLFYWQFRHCFMGVCSGKPGMVFSISRGIYSCIHGVAIEERQSVMNSPTATMNNTHSNILRLLRTAKNMANLSGVNGSPQSALDFIRRESSVYDISEHRRSFTHSDCKSYNNPPCEENLFSDYISEVERTFGNLQLKDSNVYQDHYHHHHRPHSIGSASSIDGLYDCDNPPFTTQSRSISKKPLDIGLPSSKHSQLSDLYGKFSFKSDRYSGHDDLIRSDVSDISTHTVTYGNIEGNAAKRRKQQYKDSLKKRPASAKSRREFDEIELAYRRRPPRSPDHKRYFRDKEGLRDFYLDQFRTKENSPHWEHVDLTDIYKERSDDFKRDSVSGGGPCTNRSHIKHGTGDKHGVVSGVPAPWEKNLTNVEWEDRSGGNFCRSCPSKLHNYSTTVTGQNSGRQACIRCEACKKAGNLYDISEDNSLQELDQPAAPVAVTSNASTTKYPQSPTNSKAQKKNRNKLRRQHSYDTFVDLQKEEAALAPRSVSLKDKGRFMDGSPYAHMFEMSAGESTFANNKSSVPTAGHHHHNNPGGGYMLSKSLYPDRVTQNPFIPTFGDDQCLLHGSKSYFFRQPTVAGASKARPDFRALVTNKPVVSALHGAVPARFQKDICIGNQSNPCVPNNKNPRAFNGSSNGHVYEKLSSIESDV",
# GRM1_HUMAN
"MVGLLLFFFPAIFLEVSLLPRSPGRKVLLAGASSQRSVARMDGDVIIGALFSVHHQPPAEKVPERKCGEIREQYGIQRVEAMFHTLDKINADPVLLPNITLGSEIRDSCWHSSVALEQSIEFIRDSLISIRDEKDGINRCLPDGQSLPPGRTKKPIAGVIGPGSSSVAIQVQNLLQLFDIPQIAYSATSIDLSDKTLYKYFLRVVPSDTLQARAMLDIVKRYNWTYVSAVHTEGNYGESGMDAFKELAAQEGLCIAHSDKIYSNAGEKSFDRLLRKLRERLPKARVVVCFCEGMTVRGLLSAMRRLGVVGEFSLIGSDGWADRDEVIEGYEVEANGGITIKLQSPEVRSFDDYFLKLRLDTNTRNPWFPEFWQHRFQCRLPGHLLENPNFKRICTGNESLEENYVQDSKMGFVINAIYAMAHGLQNMHHALCPGHVGLCDAMKPIDGSKLLDFLIKSSFIGVSGEEVWFDEKGDAPGRYDIMNLQYTEANRYDYVHVGTWHEGVLNIDDYKIQMNKSGVVRSVCSEPCLKGQIKVIRKGEVSCCWICTACKENEYVQDEFTCKACDLGWWPNADLTGCEPIPVRYLEWSNIESIIAIAFSCLGILVTLFVTLIFVLYRDTPVVKSSSRELCYIILAGIFLGYVCPFTLIAKPTTTSCYLQRLLVGLSSAMCYSALVTKTNRIARILAGSKKKICTRKPRFMSAWAQVIIASILISVQLTLVVTLIIMEPPMPILSYPSIKEVYLICNTSNLGVVAPLGYNGLLIMSCTYYAFKTRNVPANFNEAKYIAFTMYTTCIIWLAFVPIYFGSNYKIITTCFAVSLSVTVALGCMFTPKMYIIIAKPERNVRSAFTTSDVVRMHVGDGKLPCRSNTFLNIFRRKKAGAGNANSNGKSVSWSEPGGGQVPKGQHMWHRLSVHVKTNETACNQTAVIKPLTKSYQGSGKSLTFSDTSTKTLYNVEEEEDAQPIRFSPPGSPSMVVHRRVPSAATTPPLPSHLTAEETPLFLAEPALPKGLPPPLQQQQQPPPQQKSLMDQLQGVVSNFSTAIPDFHAVLAGPGGPGNGLRSLYPPPPPPQHLQMLPLQLSTFGEELVSPPADDDDDSERFKLLQEYVYEHEREGNTEEDELEEEEEDLQAASKLTPDDSPALTPPSPFRDSVASGSSVPSSPVSESVLCTPPNVSYASVILRDYKQSSSTL",
# SEM3F_HUMAN
"MLVAGLLLWASLLTGAWPSFPTQDHLPATPRVRLSFKELKATGTAHFFNFLLNTTDYRILLKDEDHDRMYVGSKDYVLSLDLHDINREPLIIHWAASPQRIEECVLSGKDVNGECGNFVRLIQPWNRTHLYVCGTGAYNPMCTYVNRGRRAQATPWTQTQAVRGRGSRATDGALRPMPTAPRQDYIFYLEPERLESGKGKCPYDPKLDTASALINEELYAGVYIDFMGTDAAIFRTLGKQTAMRTDQYNSRWLNDPSFIHAELIPDSAERNDDKLYFFFRERSAEAPQSPAVYARIGRICLNDDGGHCCLVNKWSTFLKARLVCSVPGEDGIETHFDELQDVFVQQTQDVRNPVIYAVFTSSGSVFRGSAVCVYSMADIRMVFNGPFAHKEGPNYQWMPFSGKMPYPRPGTCPGGTFTPSMKSTKDYPDEVINFMRSHPLMYQAVYPLQRRPLVVRTGAPYRLTTIAVDQVDAADGRYEVLFLGTDRGTVQKVIVLPKDDQELEELMLEEVEVFKDPAPVKTMTISSKRQQLYVASAVGVTHLSLHRCQAYGAACADCCLARDPYCAWDGQACSRYTASSKRRSRRQDVRHGNPIRQCRGFNSNANKNAVESVQYGVAGSAAFLECQPRSPQATVKWLFQRDPGDRRREIRAEDRFLRTEQGLLLRALQLSDRGLYSCTATENNFKHVVTRVQLHVLGRDAVHAALFPPLSMSAPPPPGAGPPTPPYQELAQLLAQPEVGLIHQYCQGYWRHVPPSPREAPGAPRSPEPQDQKKPRNRRHHPPDT",
# STX3_HUMAN
"MKDRLEQLKAKQLTQDDDTDAVEIAIDNTAFMDEFFSEIEETRLNIDKISEHVEEAKKLYSIILSAPIPEPKTKDDLEQLTTEIKKRANNVRNKLKSMEKHIEEDEVRSSADLRIRKSQHSVLSRKFVEVMTKYNEAQVDFRERSKGRIQRQLEITGKKTTDEELEEMLESGNPAIFTSGIIDSQISKQALSEIEGRHKDIVRLESSIKELHDMFMDIAMLVENQGEMLDNIELNVMHTVDHVEKARDETKKAVKYQSQARKKLIIIIVLVVVLLGILALIIGLSVGLN",
# KCAB2_HUMAN
"MYPESTTGSPARLSLRQTGSPGMIYSTRYGSPKRQLQFYRNLGKSGLRVSCLGLGTWVTFGGQITDEMAEQLMTLAYDNGINLFDTAEVYAAGKAEVVLGNIIKKKGWRRSSLVITTKIFWGGKAETERGLSRKHIIEGLKASLERLQLEYVDVVFANRPDPNTPMEETVRAMTHVINQGMAMYWGTSRWSSMEIMEAYSVARQFNLTPPICEQAEYHMFQREKVEVQLPELFHKIGVGAMTWSPLACGIVSGKYDSGIPPYSRASLKGYQWLKDKILSEEGRRQQAKLKELQAIAERLGCTLPQLAIAWCLRNEGVSSVLLGASNADQLMENIGAIQVLPKLSSSIIHEIDSILGNKPYSKKDYRS",
# ATM_HUMAN
"MSLVLNDLLICCRQLEHDRATERKKEVEKFKRLIRDPETIKHLDRHSDSKQGKYLNWDAVFRFLQKYIQKETECLRIAKPNVSASTQASRQKKMQEISSLVKYFIKCANRRAPRLKCQELLNYIMDTVKDSSNGAIYGADCSNILLKDILSVRKYWCEISQQQWLELFSVYFRLYLKPSQDVHRVLVARIIHAVTKGCCSQTDGLNSKFLDFFSKAIQCARQEKSSSGLNHILAALTIFLKTLAVNFRIRVCELGDEILPTLLYIWTQHRLNDSLKEVIIELFQLQIYIHHPKGAKTQEKGAYESTKWRSILYNLYDLLVNEISHIGSRGKYSSGFRNIAVKENLIELMADICHQVFNEDTRSLEISQSYTTTQRESSDYSVPCKRKKIELGWEVIKDHLQKSQNDFDLVPWLQIATQLISKYPASLPNCELSPLLMILSQLLPQQRHGERTPYVLRCLTEVALCQDKRSNLESSQKSDLLKLWNKIWCITFRGISSEQIQAENFGLLGAIIQGSLVEVDREFWKLFTGSACRPSCPAVCCLTLALTTSIVPGTVKMGIEQNMCEVNRSFSLKESIMKWLLFYQLEGDLENSTEVPPILHSNFPHLVLEKILVSLTMKNCKAAMNFFQSVPECEHHQKDKEELSFSEVEELFLQTTFDKMDFLTIVRECGIEKHQSSIGFSVHQNLKESLDRCLLGLSEQLLNNYSSEITNSETLVRCSRLLVGVLGCYCYMGVIAEEEAYKSELFQKAKSLMQCAGESITLFKNKTNEEFRIGSLRNMMQLCTRCLSNCTKKSPNKIASGFFLRLLTSKLMNDIADICKSLASFIKKPFDRGEVESMEDDTNGNLMEVEDQSSMNLFNDYPDSSVSDANEPGESQSTIGAINPLAEEYLSKQDLLFLDMLKFLCLCVTTAQTNTVSFRAADIRRKLLMLIDSSTLEPTKSLHLHMYLMLLKELPGEEYPLPMEDVLELLKPLSNVCSLYRRDQDVCKTILNHVLHVVKNLGQSNMDSENTRDAQGQFLTVIGAFWHLTKERKYIFSVRMALVNCLKTLLEADPYSKWAILNVMGKDFPVNEVFTQFLADNHHQVRMLAAESINRLFQDTKGDSSRLLKALPLKLQQTAFENAYLKAQEGMREMSHSAENPETLDEIYNRKSVLLTLIAVVLSCSPICEKQALFALCKSVKENGLEPHLVKKVLEKVSETFGYRRLEDFMASHLDYLVLEWLNLQDTEYNLSSFPFILLNYTNIEDFYRSCYKVLIPHLVIRSHFDEVKSIANQIQEDWKSLLTDCFPKILVNILPYFAYEGTRDSGMAQQRETATKVYDMLKSENLLGKQIDHLFISNLPEIVVELLMTLHEPANSSASQSTDLCDFSGDLDPAPNPPHFPSHVIKATFAYISNCHKTKLKSILEILSKSPDSYQKILLAICEQAAETNNVYKKHRILKIYHLFVSLLLKDIKSGLGGAWAFVLRDVIYTLIHYINQRPSCIMDVSLRSFSLCCDLLSQVCQTAVTYCKDALENHLHVIVGTLIPLVYEQVEVQKQVLDLLKYLVIDNKDNENLYITIKLLDPFPDHVVFKDLRITQQKIKYSRGPFSLLEEINHFLSVSVYDALPLTRLEGLKDLRRQLELHKDQMVDIMRASQDNPQDGIMVKLVVNLLQLSKMAINHTGEKEVLEAVGSCLGEVGPIDFSTIAIQHSKDASYTKALKLFEDKELQWTFIMLTYLNNTLVEDCVKVRSAAVTCLKNILATKTGHSFWEIYKMTTDPMLAYLQPFRTSRKKFLEVPRFDKENPFEGLDDINLWIPLSENHDIWIKTLTCAFLDSGGTKCEILQLLKPMCEVKTDFCQTVLPYLIHDILLQDTNESWRNLLSTHVQGFFTSCLRHFSQTSRSTTPANLDSESEHFFRCCLDKKSQRTMLAVVDYMRRQKRPSSGTIFNDAFWLDLNYLEVAKVAQSCAAHFTALLYAEIYADKKSMDDQEKRSLAFEEGSQSTTISSLSEKSKEETGISLQDLLLEIYRSIGEPDSLYGCGGGKMLQPITRLRTYEHEAMWGKALVTYDLETAIPSSTRQAGIIQALQNLGLCHILSVYLKGLDYENKDWCPELEELHYQAAWRNMQWDHCTSVSKEVEGTSYHESLYNALQSLRDREFSTFYESLKYARVKEVEEMCKRSLESVYSLYPTLSRLQAIGELESIGELFSRSVTHRQLSEVYIKWQKHSQLLKDSDFSFQEPIMALRTVILEILMEKEMDNSQRECIKDILTKHLVELSILARTFKNTQLPERAIFQIKQYNSVSCGVSEWQLEEAQVFWAKKEQSLALSILKQMIKKLDASCAANNPSLKLTYTECLRVCGNWLAETCLENPAVIMQTYLEKAVEVAGNYDGESSDELRNGKMKAFLSLARFSDTQYQRIENYMKSSEFENKQALLKRAKEEVGLLREHKIQTNRYTVKVQRELELDELALRALKEDRKRFLCKAVENYINCLLSGEEHDMWVFRLCSLWLENSGVSEVNGMMKRDGMKIPTYKFLPLMYQLAARMGTKMMGGLGFHEVLNNLISRISMDHPHHTLFIILALANANRDEFLTKPEVARRSRITKNVPKQSSQLDEDRTEAANRIICTIRSRRPQMVRSVEALCDAYIILANLDATQWKTQRKGINIPADQPITKLKNLEDVVVPTMEIKVDHTGEYGNLVTIQSFKAEFRLAGGVNLPKIIDCVGSDGKERRQLVKGRDDLRQDAVMQQVFQMCNTLLQRNTETRKRKLTICTYKVVPLSQRSGVLEWCTGTVPIGEFLVNNEDGAHKRYRPNDFSAFQCQKKMMEVQKKSFEEKYEVFMDVCQNFQPVFRYFCMEKFLDPAIWFEKRLAYTRSVATSSIVGYILGLGDRHVQNILINEQSAELVHIDLGVAFEQGKILPTPETVPFRLTRDIVDGMGITGVEGVFRRCCEKTMEVMRNSQETLLTIVEVLLYDPLFDWTMNPLKALYLQQRPEDETELHPTLNADDQECKRNLSDIDQSFNKVAERVLMRLQEKLKGVEEGTVLSVGGQVNLLIQQAIDPKNLSRLFPGWKAWV",
# CD5R2_HUMAN
"MGTVLSLSPASSAKGRRPGGLPEEKKKAPPAGDEALGGYGAPPVGKGGKGESRLKRPSVLISALTWKRLVAASAKKKKGSKKVTPKPASTGPDPLVQQRNRENLLRKGRDPPDGGGTAKPLAVPVPTVPAAAATCEPPSGGSAAAQPPGSGGGKPPPPPPPAPQVAPPVPGGSPRRVIVQASTGELLRCLGDFVCRRCYRLKELSPGELVGWFRGVDRSLLLQGWQDQAFITPANLVFVYLLCRESLRGDELASAAELQAAFLTCLYLAYSYMGNEISYPLKPFLVEPDKERFWQRCLRLIQRLSPQMLRLNADPHFFTQVFQDLKNEGEAAASGGGPPSGGAPAASSAARDSCAAGTKHWTMNLDR",
# PTPRS_HUMAN
"MAPTWGPGMVSVVGPMGLLVVLLVGGCAAEEPPRFIKEPKDQIGVSGGVASFVCQATGDPKPRVTWNKKGKKVNSQRFETIEFDESAGAVLRIQPLRTPRDENVYECVAQNSVGEITVHAKLTVLREDQLPSGFPNIDMGPQLKVVERTRTATMLCAASGNPDPEITWFKDFLPVDPSASNGRIKQLRSETFESTPIRGALQIESSEETDQGKYECVATNSAGVRYSSPANLYVRELREVRRVAPRFSILPMSHEIMPGGNVNITCVAVGSPMPYVKWMQGAEDLTPEDDMPVGRNVLELTDVKDSANYTCVAMSSLGVIEAVAQITVKSLPKAPGTPMVTENTATSITITWDSGNPDPVSYYVIEYKSKSQDGPYQIKEDITTTRYSIGGLSPNSEYEIWVSAVNSIGQGPPSESVVTRTGEQAPASAPRNVQARMLSATTMIVQWEEPVEPNGLIRGYRVYYTMEPEHPVGNWQKHNVDDSLLTTVGSLLEDETYTVRVLAFTSVGDGPLSDPIQVKTQQGVPGQPMNLRAEARSETSITLSWSPPRQESIIKYELLFREGDHGREVGRTFDPTTSYVVEDLKPNTEYAFRLAARSPQGLGAFTPVVRQRTLQSKPSAPPQDVKCVSVRSTAILVSWRPPPPETHNGALVGYSVRYRPLGSEDPEPKEVNGIPPTTTQILLEALEKWTQYRITTVAHTEVGPGPESSPVVVRTDEDVPSAPPRKVEAEALNATAIRVLWRSPAPGRQHGQIRGYQVHYVRMEGAEARGPPRIKDVMLADAQWETDDTAEYEMVITNLQPETAYSITVAAYTMKGDGARSKPKVVVTKGAVLGRPTLSVQQTPEGSLLARWEPPAGTAEDQVLGYRLQFGREDSTPLATLEFPPSEDRYTASGVHKGATYVFRLAARSRGGLGEEAAEVLSIPEDTPRGHPQILEAAGNASAGTVLLRWLPPVPAERNGAIVKYTVAVREAGALGPARETELPAAAEPGAENALTLQGLKPDTAYDLQVRAHTRRGPGPFSPPVRYRTFLRDQVSPKNFKVKMIMKTSVLLSWEFPDNYNSPTPYKIQYNGLTLDVDGRTTKKLITHLKPHTFYNFVLTNRGSSLGGLQQTVTAWTAFNLLNGKPSVAPKPDADGFIMVYLPDGQSPVPVQSYFIVMVPLRKSRGGQFLTPLGSPEDMDLEELIQDISRLQRRSLRHSRQLEVPRPYIAARFSVLPPTFHPGDQKQYGGFDNRGLEPGHRYVLFVLAVLQKSEPTFAASPFSDPFQLDNPDPQPIVDGEEGLIWVIGPVLAVVFIICIVIAILLYKNKPDSKRKDSEPRTKCLLNNADLAPHHPKDPVEMRRINFQTPDSGLRSPLREPGFHFESMLSHPPIPIADMAEHTERLKANDSLKLSQEYESIDPGQQFTWEHSNLEVNKPKNRYANVIAYDHSRVILQPIEGIMGSDYINANYVDGYRCQNAYIATQGPLPETFGDFWRMVWEQRSATIVMMTRLEEKSRIKCDQYWPNRGTETYGFIQVTLLDTIELATFCVRTFSLHKNGSSEKREVRQFQFTAWPDHGVPEYPTPFLAFLRRVKTCNPPDAGPIVVHCSAGVGRTGCFIVIDAMLERIKPEKTVDVYGHVTLMRSQRNYMVQTEDQYSFIHEALLEAVGCGNTEVPARSLYAYIQKLAQVEPGEHVTGMELEFKRLANSKAHTSRFISANLPCNKFKNRLVNIMPYESTRVCLQPIRGVEGSDYINASFIDGYRQQKAYIATQGPLAETTEDFWRMLWENNSTIVVMLTKLREMGREKCHQYWPAERSARYQYFVVDPMAEYNMPQYILREFKVTDARDGQSRTVRQFQFTDWPEQGVPKSGEGFIDFIGQVHKTKEQFGQDGPISVHCSAGVGRTGVFITLSIVLERMRYEGVVDIFQTVKMLRTQRPAMVQTEDEYQFCYQAALEYLGSFDHYAT",
# EIF3I_HUMAN
"MKPILLQGHERSITQIKYNREGDLLFTVAKDPIVNVWYSVNGERLGTYMGHTGAVWCVDADWDTKHVLTGSADNSCRLWDCETGKQLALLKTNSAVRTCGFDFGGNIIMFSTDKQMGYQCFVSFFDLRDPSQIDNNEPYMKIPCNDSKITSAVWGPLGECIIAGHESGELNQYSAKSGEVLVNVKEHSRQINDIQLSRDMTMFVTASKDNTAKLFDSTTLEHQKTFRTERPVNSAALSPNYDHVVLGGGQEAMDVTTTSTRIGKFEARFFHLAFEEEFGRVKGHFGPINSVAFHPDGKSYSSGGEDGYVRIHYFDPQYFEFEFEA",
# CTBP1_HUMAN
"MGSSHLLNKGLPLGVRPPIMNGPLHPRPLVALLDGRDCTVEMPILKDVATVAFCDAQSTQEIHEKVLNEAVGALMYHTITLTREDLEKFKALRIIVRIGSGFDNIDIKSAGDLGIAVCNVPAASVEETADSTLCHILNLYRRATWLHQALREGTRVQSVEQIREVASGAARIRGETLGIIGLGRVGQAVALRAKAFGFNVLFYDPYLSDGVERALGLQRVSTLQDLLFHSDCVTLHCGLNEHNHHLINDFTVKQMRQGAFLVNTARGGLVDEKALAQALKEGRIRGAALDVHESEPFSFSQGPLKDAPNLICTPHAAWYSEQASIEMREEAAREIRRAITGRIPDSLKNCVNKDHLTAATHWASMDPAVVHPELNGAAYRYPPGVVGVAPTGIPAAVEGIVPSAMSLSHGLPPVAHPPHAPSPGQTVKPEADRDHASDQL",
# AP3B2_HUMAN
"MSAAPAYSEDKGGSAGPGEPEYGHDPASGGIFSSDYKRHDDLKEMLDTNKDSLKLEAMKRIVAMIARGKNASDLFPAVVKNVACKNIEVKKLVYVYLVRYAEEQQDLALLSISTFQRGLKDPNQLIRASALRVLSSIRVPIIVPIMMLAIKEAASDMSPYVRKTAAHAIPKLYSLDSDQKDQLIEVIEKLLADKTTLVAGSVVMAFEEVCPERIDLIHKNYRKLCNLLIDVEEWGQVVIISMLTRYARTQFLSPTQNESLLEENAEKAFYGSEEDEAKGAGSEETAAAAAPSRKPYVMDPDHRLLLRNTKPLLQSRSAAVVMAVAQLYFHLAPKAEVGVIAKALVRLLRSHSEVQYVVLQNVATMSIKRRGMFEPYLKSFYIRSTDPTQIKILKLEVLTNLANETNIPTVLREFQTYIRSMDKDFVAATIQAIGRCATNIGRVRDTCLNGLVQLLSNRDELVVAESVVVIKKLLQMQPAQHGEIIKHLAKLTDNIQVPMARASILWLIGEYCEHVPRIAPDVLRKMAKSFTAEEDIVKLQVINLAAKLYLTNSKQTKLLTQYVLSLAKYDQNYDIRDRARFTRQLIVPSEQGGALSRHAKKLFLAPKPAPVLESSFKDRDHFQLGSLSHLLNAKATGYQELPDWPEEAPDPSVRNVEVPEWTKCSNREKRKEKEKPFYSDSEGESGPTESADSDPESESESDSKSSSESGSGESSSESDNEDQDEDEEKGRGSESEQSEEDGKRKTKKKVPERKGEASSSDEGSDSSSSSSESEMTSESEEEQLEPASWSRKTPPSSKSAPATKEISLLDLEDFTPPSVQPVSPPAIVSTSLAADLEGLTLTDSTLVPSLLSPVSGVGRQELLHRVAGEGLAVDYTFSRQPFSGDPHMVSVHIHFSNSSDTPIKGLHVGTPKLPAGISIQEFPEIESLAPGESATAVMGINFCDSTQAANFQLCTQTRQFYVSIQPPVGELMAPVFMSENEFKKEQGKLMGMNEITEKLMLPDTCRSDHIVVQKVTATANLGRVPCGTSDEYRFAGRTLTGGSLVLLTLDARPAGAAQLTVNSEKMVIGTMLVKDVIQALTQ",
# MPP3_HUMAN
"MPVLSEDSGLHETLALLTSQLRPDSNHKEEMGFLRDVFSEKSLSYLMKIHEKLRYYERQSPTPVLHSAVALAEDVMEELQAASVHSDERELLQLLSTPHLRAVLMVHDTVAQKNFDPVLPPLPDNIDEDFDEESVKIVRLVKNKEPLGATIRRDEHSGAVVVARIMRGGAADRSGLVHVGDELREVNGIAVLHKRPDEISQILAQSQGSITLKIIPATQEEDRLKESKVFMRALFHYNPREDRAIPCQEAGLPFQRRQVLEVVSQDDPTWWQAKRVGDTNLRAGLIPSKGFQERRLSYRRAAGTLPSPQSLRKPPYDQPCDKETCDCEGYLKGHYVAGLRRSFRLGCRERLGGSQEGKMSSGAESPELLTYEEVARYQHQPGERPRLVVLIGSLGARLHELKQKVVAENPQHFGVAVPHTTRPRKSHEKEGVEYHFVSKQAFEADLHHNKFLEHGEYKENLYGTSLEAIQAVMAKNKVCLVDVEPEALKQLRTSEFKPYIIFVKPAIQEKRKTPPMSPACEDTAAPFDEQQQEMAASAAFIDRHYGHLVDAVLVKEDLQGAYSQLKVVLEKLSKDTHWVPVSWVR",
# PLD1_HUMAN
"MSLKNEPRVNTSALQKIAADMSNIIENLDTRELHFEGEEVDYDVSPSDPKIQEVYIPFSAIYNTQGFKEPNIQTYLSGCPIKAQVLEVERFTSTTRVPSINLYTIELTHGEFKWQVKRKFKHFQEFHRELLKYKAFIRIPIPTRRHTFRRQNVREEPREMPSLPRSSENMIREEQFLGRRKQLEDYLTKILKMPMYRNYHATTEFLDISQLSFIHDLGPKGIEGMIMKRSGGHRIPGLNCCGQGRACYRWSKRWLIVKDSFLLYMKPDSGAIAFVLLVDKEFKIKVGKKETETKYGIRIDNLSRTLILKCNSYRHARWWGGAIEEFIQKHGTNFLKDHRFGSYAAIQENALAKWYVNAKGYFEDVANAMEEANEEIFITDWWLSPEIFLKRPVVEGNRWRLDCILKRKAQQGVRIFIMLYKEVELALGINSEYTKRTLMRLHPNIKVMRHPDHVSSTVYLWAHHEKLVIIDQSVAFVGGIDLAYGRWDDNEHRLTDVGSVKRVTSGPSLGSLPPAAMESMESLRLKDKNEPVQNLPIQKSIDDVDSKLKGIGKPRKFSKFSLYKQLHRHHLHDADSISSIDSTSSYFNHYRSHHNLIHGLKPHFKLFHPSSESEQGLTRPHADTGSIRSLQTGVGELHGETRFWHGKDYCNFVFKDWVQLDKPFADFIDRYSTPRMPWHDIASAVHGKAARDVARHFIQRWNFTKIMKSKYRSLSYPFLLPKSQTTAHELRYQVPGSVHANVQLLRSAADWSAGIKYHEESIHAAYVHVIENSRHYIYIENQFFISCADDKVVFNKIGDAIAQRILKAHRENQKYRVYVVIPLLPGFEGDISTGGGNALQAIMHFNYRTMCRGENSILGQLKAELGNQWINYISFCGLRTHAELEGNLVTELIYVHSKLLIADDNTVIIGSANINDRSMLGKRDSEMAVIVQDTETVPSVMDGKEYQAGRFARGLRLQCFRVVLGYLDDPSEDIQDPVSDKFFKEVWVSTAARNATIYDKVFRCLPNDEVHNLIQLRDFINKPVLAKEDPIRAEEELKKIRGFLVQFPFYFLSEESLLPSVGTKEAIVPMEVWT",
# UB2V1_HUMAN
"MAATTGSGVKVPRNFRLLEELEEGQKGVGDGTVSWGLEDDEDMTLTRWTGMIIGPPRTIYENRIYSLKIECGPKYPEAPPFVRFVTKINMNGVNSSNGVVDPRAISVLAKWQNSYSIKVVLQELRRLMMSKENMKLPQPPEGQCYSN",
# LSAMP_HUMAN
"MVRRVQPDRKQLPLVLLRLLCLLPTGLPVRSVDFNRGTDNITVRQGDTAILRCVVEDKNSKVAWLNRSGIIFAGHDKWSLDPRVELEKRHSLEYSLRIQKVDVYDEGSYTCSVQTQHEPKTSQVYLIVQVPPKISNISSDVTVNEGSNVTLVCMANGRPEPVITWRHLTPTGREFEGEEEYLEILGITREQSGKYECKAANEVSSADVKQVKVTVNYPPTITESKSNEATTGRQASLKCEASAVPAPDFEWYRDDTRINSANGLEIKSTEGQSSLTVTNVTEEHYGNYTCVAANKLGVTNASLVLFRPGSVRGINGSISLAVPLWLLAASLLCLLSKC",
# MYO9B_HUMAN
"MSVKEAGSSGRREQAAYHLHIYPQLSTTESQASCRVTATKDSTTSDVIKDAIASLRLDGTKCYVLVEVKESGGEEWVLDANDSPVHRVLLWPRRAQDEHPQEDGYYFLLQERNADGTIKYVHMQLVAQATATRRLVERGLLPRQQADFDDLCNLPELTEGNLLKNLKHRFLQQKIYTYAGSILVAINPFKFLPIYNPKYVKMYENQQLGKLEPHVFALADVAYYTMLRKRVNQCIVISGESGSGKTQSTNFLIHCLTALSQKGYASGVERTILGAGPVLEAFGNAKTAHNNNSSRFGKFIQVSYLESGIVRGAVVEKYLLEKSRLVSQEKDERNYHVFYYLLLGVSEEERQEFQLKQPEDYFYLNQHNLKIEDGEDLKHDFERLKQAMEMVGFLPATKKQIFAVLSAILYLGNVTYKKRATGREEGLEVGPPEVLDTLSQLLKVKREILVEVLTKRKTVTVNDKLILPYSLSEAITARDSMAKSLYSALFDWIVLRINHALLNKKDVEEAVSCLSIGVLDIFGFEDFERNSFEQFCINYANEQLQYYFNQHIFKLEQEEYQGEGITWHNIGYTDNVGCIHLISKKPTGLFYLLDEESNFPHATSQTLLAKFKQQHEDNKYFLGTPVMEPAFIIQHFAGKVKYQIKDFREKNMDYMRPDIVALLRGSDSSYVRELIGMDPVAVFRWAVLRAAIRAMAVLREAGRLRAERAEKAAGMSSPGAQSHPEELPRGASTPSEKLYRDLHNQMIKSIKGLPWQGEDPRSLLQSLSRLQKPRAFILKSKGIKQKQIIPKNLLDSKSLKLIISMTLHDRTTKSLLHLHKKKKPPSISAQFQTSLNKLLEALGKAEPFFIRCIRSNAEKKELCFDDELVLQQLRYTGMLETVRIRRSGYSAKYTFQDFTEQFQVLLPKDAQPCREVISTLLEKMKIDKRNYQIGKTKVFLKETERQALQETLHREVVRKILLLQSWFRMVLERRHFLQMKRAAVTIQACWRSYRVRRALERTQAAVYLQASWRGYWQRKLYRHQKQSIIRLQSLCRGHLQRKSFSQMISEKQKAEEKEREALEAARAGAEEGGQGQAAGGQQVAEQGPEPAEDGGHLASEPEVQPSDRSPLEHSSPEKEAPSPEKTLPPQKTVAAESHEKVPSSREKRESRRQRGLEHVKFQNKHIQSCKEESALREPSRRVTQEQGVSLLEDKKESREDETLLVVETEAENTSQKQPTEQPQAMAVGKVSEETEKTLPSGSPRPGQLERPTSLALDSRVSPPAPGSAPETPEDKSKPCGSPRVQEKPDSPGGSTQIQRYLDAERLASAVELWRGKKLVAAASPSAMLSQSLDLSDRHRATGAALTPTEERRTSFSTSDVSKLLPSLAKAQPAAETTDGERSAKKPAVQKKKPGDASSLPDAGLSPGSQVDSKSTFKRLFLHKTKDKKYSLEGAEELENAVSGHVVLEATTMKKGLEAPSGQQHRHAAGEKRTKEPGGKGKKNRNVKIGKITVSEKWRESVFRQITNANELKYLDEFLLNKINDLRSQKTPIESLFIEATEKFRSNIKTMYSVPNGKIHVGYKDLMENYQIVVSNLATERGQKDTNLVLNLFQSLLDEFTRGYTKNDFEPVKQSKAQKKKRKQERAVQEHNGHVFASYQVSIPQSCEQCLSYIWLMDKALLCSVCKMTCHKKCVHKIQSHCSYTYGRKGEPGVEPGHFGVCVDSLTSDKASVPIVLEKLLEHVEMHGLYTEGLYRKSGAANRTRELRQALQTDPAAVKLENFPIHAITGVLKQWLRELPEPLMTFAQYGDFLRAVELPEKQEQLAAIYAVLEHLPEANHNSLERLIFHLVKVALLEDVNRMSPGALAIIFAPCLLRCPDNSDPLTSMKDVLKITTCVEMLIKEQMRKYKVKMEEISQLEAAESIAFRRLSLLRQNAPWPLKLGFSSPYEGVLNKSPKTRDIQEEELEVLLEEEAAGGDEDREKEILIERIQSIKEEKEDITYRLPELDPRGSDEENLDSETSASTESLLEERAGRGASEGPPAPALPCPGAPTPSPLPTVAAPPRRRPSSFVTVRVKTPRRTPIMPTANIKLPPGLPSHLPRWAPGAREAAAPVRRREPPARRPDQIHSVYITPGADLPVQGALEPLEEDGQPPGAKRRYSDPPTYCLPPASGQTNG",
# ROCK1_HUMAN
"MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPALRKNKNIDNFLSRYKDTINKIRDLRMKAEDYEVVKVIGRGAFGEVQLVRHKSTRKVYAMKLLSKFEMIKRSDSAFFWEERDIMAFANSPWVVQLFYAFQDDRYLYMVMEYMPGGDLVNLMSNYDVPEKWARFYTAEVVLALDAIHSMGFIHRDVKPDNMLLDKSGHLKLADFGTCMKMNKEGMVRCDTAVGTPDYISPEVLKSQGGDGYYGRECDWWSVGVFLYEMLVGDTPFYADSLVGTYSKIMNHKNSLTFPDDNDISKEAKNLICAFLTDREVRLGRNGVEEIKRHLFFKNDQWAWETLRDTVAPVVPDLSSDIDTSNFDDLEEDKGEEETFPIPKAFVGNQLPFVGFTYYSNRRYLSSANPNDNRTSSNADKSLQESLQKTIYKLEEQLHNEMQLKDEMEQKCRTSNIKLDKIMKELDEEGNQRRNLESTVSQIEKEKMLLQHRINEYQRKAEQENEKRRNVENEVSTLKDQLEDLKKVSQNSQLANEKLSQLQKQLEEANDLLRTESDTAVRLRKSHTEMSKSISQLESLNRELQERNRILENSKSQTDKDYYQLQAILEAERRDRGHDSEMIGDLQARITSLQEEVKHLKHNLEKVEGERKEAQDMLNHSEKEKNNLEIDLNYKLKSLQQRLEQEVNEHKVTKARLTDKHQSIEEAKSVAMCEMEKKLKEEREAREKAENRVVQIEKQCSMLDVDLKQSQQKLEHLTGNKERMEDEVKNLTLQLEQESNKRLLLQNELKTQAFEADNLKGLEKQMKQEINTLLEAKRLLEFELAQLTKQYRGNEGQMRELQDQLEAEQYFSTLYKTQVKELKEEIEEKNRENLKKIQELQNEKETLATQLDLAETKAESEQLARGLLEEQYFELTQESKKAASRNRQEITDKDHTVSRLEEANSMLTKDIEILRRENEELTEKMKKAEEEYKLEKEEEISNLKAAFEKNINTERTLKTQAVNKLAEIMNRKDFKIDRKKANTQDLRKKEKENRKLQLELNQEREKFNQMVVKHQKELNDMQAQLVEECAHRNELQMQLASKESDIEQLRAKLLDLSDSTSVASFPSADETDGNLPESRIEGWLSVPNRGNIKRYGWKKQYVVVSSKKILFYNDEQDKEQSNPSMVLDIDKLFHVRPVTQGDVYRAETEEIPKIFQILYANEGECRKDVEMEPVQQAEKTNFQNHKGHEFIPTLYHFPANCDACAKPLWHVFKPPPALECRRCHVKCHRDHLDKKEDLICPCKVSYDVTSARDMLLLACSQDEQKKWVTHLVKKIPKNPPSGFVRASPRTLSTRSTANQSFRKVVKNTSGKTS",
# FZD5_HUMAN
"MARPDPSAPPSLLLLLLAQLVGRAAAASKAPVCQEITVPMCRGIGYNLTHMPNQFNHDTQDEAGLEVHQFWPLVEIQCSPDLRFFLCSMYTPICLPDYHKPLPPCRSVCERAKAGCSPLMRQYGFAWPERMSCDRLPVLGRDAEVLCMDYNRSEATTAPPRPFPAKPTLPGPPGAPASGGECPAGGPFVCKCREPFVPILKESHPLYNKVRTGQVPNCAVPCYQPSFSADERTFATFWIGLWSVLCFISTSTTVATFLIDMERFRYPERPIIFLSACYLCVSLGFLVRLVVGHASVACSREHNHIHYETTGPALCTIVFLLVYFFGMASSIWWVILSLTWFLAAGMKWGNEAIAGYAQYFHLAAWLIPSVKSITALALSSVDGDPVAGICYVGNQNLNSLRGFVLGPLVLYLLVGTLFLLAGFVSLFRIRSVIKQGGTKTDKLEKLMIRIGIFTLLYTVPASIVVACYLYEQHYRESWEAALTCACPGHDTGQPRAKPEYWVLMLKYFMCLVVGITSGVWIWSGKTVESWRRFTSRCCCRPRRGHKSGGAMAAGDYPEASAALTGRTGPPGPAATYHKQVSLSHV",
# DRP2_HUMAN
"MQPMVMQGCPYTLPRCHDWQAADQFHHSSSLRSTCPHPQVRAAVTSPAPPQDGAGVPCLSLKLLNGSVGASGPLEPPAMNLCWNEIKKKSHNLRARLEAFSDHSGKLQLPLQEIIDWLSQKDEELSAQLPLQGDVALVQQEKETHAAFMEEVKSRGPYIYSVLESAQAFLSQHPFEELEEPHSESKDTSPKQRIQNLSRFVWKQATVASELWEKLTARCVDQHRHIERTLEQLLEIQGAMEELSTTLSQAEGVRATWEPIGDLFIDSLPEHIQAIKLFKEEFSPMKDGVKLVNDLAHQLAISDVHLSMENSQALEQINVRWKQLQASVSERLKQLQDAHRDFGPGSQHFLSSSVQVPWERAISPNKVPYYINHQAQTTCWDHPKMTELYQTLADLNNIKFSAYRTAMKLRRVQKALRLDLVTLTTALEIFNEHDLQASEHVMDVVEVIHCLTALYERLEEERGILVNVPLCVDMSLNWLLNVFDSGRSGKMRALSFKTGIACLCGTEVKEKLQYLFSQVANSGSQCDQRHLGVLLHEAIQVPRQLGEVAAFGGSNVEPSVRSCFRFSTGKPVIEASQFLEWVNLEPQSMVWLAVLHRVTIAEQVKHQTKCSICRQCPIKGFRYRSLKQFNVDICQTCFLTGRASKGNKLHYPIMEYYTPTTSSENMRDFATTLKNKFRSKHYFSKHPQRGYLPVQSVLEADYSETPASSPMWPHADTHSRIEHFASRLAEMESQNCSFFNDSLSPDDSIDEDQYLLRHSSPITDREPAFGQQAPCSVATESKGELQKILAHLEDENRILQGELRRLKWQHEEAAEAPSLADGSTEAATDHRNEELLAEARILRQHKSRLETRMQILEDHNKQLESQLQRLRELLLQPPTESDGSGSAGSSLASSPQQSEGSHPREKGQTTPDTEAADDVGSKSQDVSLCLEDIMEKLRHAFPSVRSSDVTANTLLAS",
# PICAL_HUMAN
"MSGQSLTDRITAAQHSVTGSAVSKTVCKATTHEIMGPKKKHLDYLIQCTNEMNVNIPQLADSLFERTTNSSWVVVFKSLITTHHLMVYGNERFIQYLASRNTLFNLSNFLDKSGLQGYDMSTFIRRYSRYLNEKAVSYRQVAFDFTKVKRGADGVMRTMNTEKLLKTVPIIQNQMDALLDFNVNSNELTNGVINAAFMLLFKDAIRLFAAYNEGIINLLEKYFDMKKNQCKEGLDIYKKFLTRMTRISEFLKVAEQVGIDRGDIPDLSQAPSSLLDALEQHLASLEGKKIKDSTAASRATTLSNAVSSLASTGLSLTKVDEREKQAALEEEQARLKALKEQRLKELAKKPHTSLTTAASPVSTSAGGIMTAPAIDIFSTPSSSNSTSKLPNDLLDLQQPTFHPSVHPMSTASQVASTWGDPFSATVDAVDDAIPSLNPFLTKSSGDVHLSISSDVSTFTTRTPTHEMFVGFTPSPVAQPHPSAGLNVDFESVFGNKSTNVIVDSGGFDELGGLLKPTVASQNQNLPVAKLPPSKLVSDDLDSSLANLVGNLGIGNGTTKNDVNWSQPGEKKLTGGSNWQPKVAPTTAWNAATMAPPVMAYPATTPTGMIGYGIPPQMGSVPVMTQPTLIYSQPVMRPPNPFGPVSGAQIQFM",
# SQSTM_HUMAN
"MASLTVKAYLLGKEDAAREIRRFSFCCSPEPEAEAEAAAGPGPCERLLSRVAALFPALRPGGFQAHYRDEDGDLVAFSSDEELTMAMSYVKDDIFRIYIKEKKECRRDHRPPCAQEAPRNMVHPNVICDGCNGPVVGTRYKCSVCPDYDLCSVCEGKGLHRGHTKLAFPSPFGHLSEGFSHSRWLRKVKHGHFGWPGWEMGPPGNWSPRPPRAGEARPGPTAESASGPSEDPSVNFLKNVGESVAAALSPLGIEVDIDVEHGGKRSRLTPVSPESSSTEEKSSSQPSSCCSDPSKPGGNVEGATQSLAEQMRKIALESEGRPEEQMESDNCSGGDDDWTHLSSKEVDPSTGELQSLQMPESEGPSSLDPSQEGPTGLKEAALYPHLPPEADPRLIESLSQMLSMGFSDEGGWLTRLLQTKNYDIGAALDTIQYSKHPPPL",
# PIN1_HUMAN
"MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGKNGQGEPARVRCSHLLVKHSQSRRPSSWRQEKITRTKEEALELINGYIQKIKSGEEDFESLASQFSDCSSAKARGDLGAFSRGQMQKPFEDASFALRTGEMSGPVFTDSGIHIILRTE",
# ATR_HUMAN
"MGEHGLELASMIPALRELGSATPEEYNTVVQKPRQILCQFIDRILTDVNVVAVELVKKTDSQPTSVMLLDFIQHIMKSSPLMFVNVSGSHEAKGSCIEFSNWIITRLLRIAATPSCHLLHKKICEVICSLLFLFKSKSPAIFGVLTKELLQLFEDLVYLHRRNVMGHAVEWPVVMSRFLSQLDEHMGYLQSAPLQLMSMQNLEFIEVTLLMVLTRIIAIVFFRRQELLLWQIGCVLLEYGSPKIKSLAISFLTELFQLGGLPAQPASTFFSSFLELLKHLVEMDTDQLKLYEEPLSKLIKTLFPFEAEAYRNIEPVYLNMLLEKLCVMFEDGVLMRLKSDLLKAALCHLLQYFLKFVPAGYESALQVRKVYVRNICKALLDVLGIEVDAEYLLGPLYAALKMESMEIIEEIQCQTQQENLSSNSDGISPKRRRLSSSLNPSKRAPKQTEEIKHVDMNQKSILWSALKQKAESLQISLEYSGLKNPVIEMLEGIAVVLQLTALCTVHCSHQNMNCRTFKDCQHKSKKKPSVVITWMSLDFYTKVLKSCRSLLESVQKLDLEATIDKVVKIYDALIYMQVNSSFEDHILEDLCGMLSLPWIYSHSDDGCLKLTTFAANLLTLSCRISDSYSPQAQSRCVFLLTLFPRRIFLEWRTAVYNWALQSSHEVIRASCVSGFFILLQQQNSCNRVPKILIDKVKDDSDIVKKEFASILGQLVCTLHGMFYLTSSLTEPFSEHGHVDLFCRNLKATSQHECSSSQLKASVCKPFLFLLKKKIPSPVKLAFIDNLHHLCKHLDFREDETDVKAVLGTLLNLMEDPDKDVRVAFSGNIKHILESLDSEDGFIKELFVLRMKEAYTHAQISRNNELKDTLILTTGDIGRAAKGDLVPFALLHLLHCLLSKSASVSGAAYTEIRALVAAKSVKLQSFFSQYKKPICQFLVESLHSSQMTALPNTPCQNADVRKQDVAHQREMALNTLSEIANVFDFPDLNRFLTRTLQVLLPDLAAKASPAASALIRTLGKQLNVNRREILINNFKYIFSHLVCSCSKDELERALHYLKNETEIELGSLLRQDFQGLHNELLLRIGEHYQQVFNGLSILASFASSDDPYQGPRDIISPELMADYLQPKLLGILAFFNMQLLSSSVGIEDKKMALNSLMSLMKLMGPKHVSSVRVKMMTTLRTGLRFKDDFPELCCRAWDCFVRCLDHACLGSLLSHVIVALLPLIHIQPKETAAIFHYLIIENRDAVQDFLHEIYFLPDHPELKKIKAVLQEYRKETSESTDLQTTLQLSMKAIQHENVDVRIHALTSLKETLYKNQEKLIKYATDSETVEPIISQLVTVLLKGCQDANSQARLLCGECLGELGAIDPGRLDFSTTETQGKDFTFVTGVEDSSFAYGLLMELTRAYLAYADNSRAQDSAAYAIQELLSIYDCREMETNGPGHQLWRRFPEHVREILEPHLNTRYKSSQKSTDWSGVKKPIYLSKLGSNFAEWSASWAGYLITKVRHDLASKIFTCCSIMMKHDFKVTIYLLPHILVYVLLGCNQEDQQEVYAEIMAVLKHDDQHTINTQDIASDLCQLSTQTVFSMLDHLTQWARHKFQALKAEKCPHSKSNRNKVDSMVSTVDYEDYQSVTRFLDLIPQDTLAVASFRSKAYTRAVMHFESFITEKKQNIQEHLGFLQKLYAAMHEPDGVAGVSAIRKAEPSLKEQILEHESLGLLRDATACYDRAIQLEPDQIIHYHGVVKSMLGLGQLSTVITQVNGVHANRSEWTDELNTYRVEAAWKLSQWDLVENYLAADGKSTTWSVRLGQLLLSAKKRDITAFYDSLKLVRAEQIVPLSAASFERGSYQRGYEYIVRLHMLCELEHSIKPLFQHSPGDSSQEDSLNWVARLEMTQNSYRAKEPILALRRALLSLNKRPDYNEMVGECWLQSARVARKAGHHQTAYNALLNAGESRLAELYVERAKWLWSKGDVHQALIVLQKGVELCFPENETPPEGKNMLIHGRAMLLVGRFMEETANFESNAIMKKYKDVTACLPEWEDGHFYLAKYYDKLMPMVTDNKMEKQGDLIRYIVLHFGRSLQYGNQFIYQSMPRMLTLWLDYGTKAYEWEKAGRSDRVQMRNDLGKINKVITEHTNYLAPYQFLTAFSQLISRICHSHDEVFVVLMEIIAKVFLAYPQQAMWMMTAVSKSSYPMRVNRCKEILNKAIHMKKSLEKFVGDATRLTDKLLELCNKPVDGSSSTLSMSTHFKMLKKLVEEATFSEILIPLQSVMIPTLPSILGTHANHASHEPFPGHWAYIAGFDDMVEILASLQKPKKISLKGSDGKFYIMMCKPKDDLRKDCRLMEFNSLINKCLRKDAESRRRELHIRTYAVIPLNDECGIIEWVNNTAGLRPILTKLYKEKGVYMTGKELRQCMLPKSAALSEKLKVFREFLLPRHPPIFHEWFLRTFPDPTSWYSSRSAYCRSTAVMSMVGYILGLGDRHGENILFDSLTGECVHVDFNCLFNKGETFEVPEIVPFRLTHNMVNGMGPMGTEGLFRRACEVTMRLMRDQREPLMSVLKTFLHDPLVEWSKPVKGHSKAPLNETGEVVNEKAKTHVLDIEQRLQGVIKTRNRVTGLPLSIEGHVHYLIQEATDENLLCQMYLGWTPYM",
# 4EBP1_HUMAN
"MSGGSSCSQTPSRAIPATRRVVLGDGVQLPPGDYSTTPGGTLFSTTPGGTRIIYDRKFLMECRNSPVTKTPPRDLPTIPGVTSPSSDEPPMEASQSHLRNSPEDKRAGGEESQFEMDI",
# 4EBP2_HUMAN
"MSSSAGSGHQPSQSRAIPTRTVAISDAAQLPHDYCTTPGGTLFSTTPGGTRIIYDRKFLLDRRNSPMAQTPPCHLPNIPGVTSPGTLIEDSKVEVNNLNNLNNHDRKHAVGDDAQFEMDI",
# KCC2B_HUMAN
"MATTVTCTRFTDEYQLYEDIGKGAFSVVRRCVKLCTGHEYAAKIINTKKLSARDHQKLEREARICRLLKHSNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIQQILEAVLHCHQMGVVHRDLKPENLLLASKCKGAAVKLADFGLAIEVQGDQQAWFGFAGTPGYLSPEVLRKEAYGKPVDIWACGVILYILLVGYPPFWDEDQHKLYQQIKAGAYDFPSPEWDTVTPEAKNLINQMLTINPAKRITAHEALKHPWVCQRSTVASMMHRQETVECLKKFNARRKLKGAILTTMLATRNFSVGRQTTAPATMSTAASGTTMGLVEQAKSLLNKKADGVKPQTNSTKNSAAATSPKGTLPPAALEPQTTVIHNPVDGIKESSDSANTTIEDEDAKAPRVPDILSSVRRGSGAPEAEGPLPCPSPAPFSPLPAPSPRISDILNSVRRGSGTPEAEGPLSAGPPPCLSPALLGPLSSPSPRISDILNSVRRGSGTPEAEGPSPVGPPPCPSPTIPGPLPTPSRKQEIIKTTEQLIEAVNNGDFEAYAKICDPGLTSFEPEALGNLVEGMDFHRFYFENLLAKNSKPIHTTILNPHVHVIGEDAACIAYIRLTQYIDGQGRPRTSQSEETRVWHRRDGKWQNVHFHCSGAPVAPLQ",
# KCC2G_HUMAN
"MATTATCTRFTDDYQLFEELGKGAFSVVRRCVKKTSTQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIHQILESVNHIHQHDIVHRDLKPENLLLASKCKGAAVKLADFGLAIEVQGEQQAWFGFAGTPGYLSPEVLRKDPYGKPVDIWACGVILYILLVGYPPFWDEDQHKLYQQIKAGAYDFPSPEWDTVTPEAKNLINQMLTINPAKRITADQALKHPWVCQRSTVASMMHRQETVECLRKFNARRKLKGAILTTMLVSRNFSAAKSLLNKKSDGGVKKRKSSSSVHLMPQSNNKNSLVSPAQEPAPLQTAMEPQTTVVHNATDGIKGSTESCNTTTEDEDLKGRVPEGRSSRDRTAPSAGMQPQPSLCSSAMRKQEIIKITEQLIEAINNGDFEAYTKICDPGLTSFEPEALGNLVEGMDFHKFYFENLLSKNSKPIHTTILNPHVHVIGEDAACIAYIRLTQYIDGQGRPRTSQSEETRVWHRRDGKWLNVHYHCSGAPAAPLQ",
# KCC2D_HUMAN
"MASTTTCTRFTDEYQLFEELGKGAFSVVRRCMKIPTGQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIQQILESVNHCHLNGIVHRDLKPENLLLASKSKGAAVKLADFGLAIEVQGDQQAWFGFAGTPGYLSPEVLRKDPYGKPVDMWACGVILYILLVGYPPFWDEDQHRLYQQIKAGAYDFPSPEWDTVTPEAKDLINKMLTINPAKRITASEALKHPWICQRSTVASMMHRQETVDCLKKFNARRKLKGAILTTMLATRNFSAAKSLLKKPDGVKESTESSNTTIEDEDVKARKQEIIKVTEQLIEAINNGDFEAYTKICDPGLTAFEPEALGNLVEGMDFHRFYFENALSKSNKPIHTIILNPHVHLVGDDAACIAYIRLTQYMDGSGMPKTMQSEETRVWHRRDGKWQNVHFHRSGSPTVPIKPPCIPNGKENFSGGTSLWQNI",
# ULA1_HUMAN
"MAQLGKLLKEQKYDRQLRLWGDHGQEALESAHVCLINATATGTEILKNLVLPGIGSFTIIDGNQVSGEDAGNNFFLQRSSIGKNRAEAAMEFLQELNSDVSGSFVEESPENLLDNDPSFFCRFTVVVATQLPESTSLRLADVLWNSQIPLLICRTYGLVGYMRIIIKEHPVIESHPDNALEDLRLDKPFPELREHFQSYDLDHMEKKDHSHTPWIVIIAKYLAQWYSETNGRIPKTYKEKEDFRDLIRQGILKNENGAPEDEENFEEAIKNVNTALNTTQIPSSIEDIFNDDRCINITKQTPSFWILARALKEFVAKEGQGNLPVRGTIPDMIADSGKYIKLQNVYREKAKKDAAAVGNHVAKLLQSIGQAPESISEKELKLLCSNSAFLRVVRCRSLAEEYGLDTINKDEIISSMDNPDNEIVLYLMLRAVDRFHKQQGRYPGVSNYQVEEDIGKLKSCLTGFLQEYGLSVMVKDDYVHEFCRYGAAEPHTIAAFLGGAAAQEVIKIITKQFVIFNNTYIYSGMSQTSATFQL",
# DGKZ_HUMAN
"MEPRDGSPEARSSDSESASASSSGSERDAGPEPDKAPRRLNKRRFPGLRLFGHRKAITKSGLQHLAPPPPTPGAPCSESERQIRSTVDWSESATYGEHIWFETNVSGDFCYVGEQYCVARMLKSVSRRKCAACKIVVHTPCIEQLEKINFRCKPSFRESGSRNVREPTFVRHHWVHRRRQDGKCRHCGKGFQQKFTFHSKEIVAISCSWCKQAYHSKVSCFMLQQIEEPCSLGVHAAVVIPPTWILRARRPQNTLKASKKKKRASFKRKSSKKGPEEGRWRPFIIRPTPSPLMKPLLVFVNPKSGGNQGAKIIQSFLWYLNPRQVFDLSQGGPKEALEMYRKVHNLRILACGGDGTVGWILSTLDQLRLKPPPPVAILPLGTGNDLARTLNWGGGYTDEPVSKILSHVEEGNVVQLDRWDLHAEPNPEAGPEDRDEGATDRLPLDVFNNYFSLGFDAHVTLEFHESREANPEKFNSRFRNKMFYAGTAFSDFLMGSSKDLAKHIRVVCDGMDLTPKIQDLKPQCVVFLNIPRYCAGTMPWGHPGEHHDFEPQRHDDGYLEVIGFTMTSLAALQVGGHGERLTQCREVVLTTSKAIPVQVDGEPCKLAASRIRIALRNQATMVQKAKRRSAAPLHSDQQPVPEQLRIQVSRVSMHDYEALHYDKEQLKEASVPLGTVVVPGDSDLELCRAHIERLQQEPDGAGAKSPTCQKLSPKWCFLDATTASRFYRIDRAQEHLNYVTEIAQDEIYILDPELLGASARPDLPTPTSPLPTSPCSPTPRSLQGDAAPPQGEELIEAAKRNDFCKLQELHRAGGDLMHRDEQSRTLLHHAVSTGSKDVVRYLLDHAPPEILDAVEENGETCLHQAAALGQRTICHYIVEAGASLMKTDQQGDTPRQRAEKAQDTELAAYLENRQHYQMIQREDQETAV",
# SNX1_HUMAN
"MASGGGGCSASERLPPPFPGLEPESEGAAGGSEPEAGDSDTEGEDIFTGAAVVSKHQSPKITTSLLPINNGSKENGIHEEQDQEPQDLFADATVELSLDSTQNNQKKVLAKTLISLPPQEATNSSKPQPTYEELEEEEQEDQFDLTVGITDPEKIGDGMNAYVAYKVTTQTSLPLFRSKQFAVKRRFSDFLGLYEKLSEKHSQNGFIVPPPPEKSLIGMTKVKVGKEDSSSAEFLEKRRAALERYLQRIVNHPTMLQDPDVREFLEKEELPRAVGTQTLSGAGLLKMFNKATDAVSKMTIKMNESDIWFEEKLQEVECEEQRLRKLHAVVETLVNHRKELALNTAQFAKSLAMLGSSEDNTALSRALSQLAEVEEKIEQLHQEQANNDFFLLAELLSDYIRLLAIVRAAFDQRMKTWQRWQDAQATLQKKREAEARLLWANKPDKLQQAKDEILEWESRVTQYERDFERISTVVRKEVIRFEKEKSKDFKNHVIKYLETLLYSQQQLAKYWEAFLPEAKAIS",
# CUL3_HUMAN
"MSNLSKGTGSRKDTKMRIRAFPMTMDEKYVNSIWDLLKNAIQEIQRKNNSGLSFEELYRNAYTMVLHKHGEKLYTGLREVVTEHLINKVREDVLNSLNNNFLQTLNQAWNDHQTAMVMIRDILMYMDRVYVQQNNVENVYNLGLIIFRDQVVRYGCIRDHLRQTLLDMIARERKGEVVDRGAIRNACQMLMILGLEGRSVYEEDFEAPFLEMSAEFFQMESQKFLAENSASVYIKKVEARINEEIERVMHCLDKSTEEPIVKVVERELISKHMKTIVEMENSGLVHMLKNGKTEDLGCMYKLFSRVPNGLKTMCECMSSYLREQGKALVSEEGEGKNPVDYIQGLLDLKSRFDRFLLESFNNDRLFKQTIAGDFEYFLNLNSRSPEYLSLFIDDKLKKGVKGLTEQEVETILDKAMVLFRFMQEKDVFERYYKQHLARRLLTNKSVSDDSEKNMISKLKTECGCQFTSKLEGMFRDMSISNTTMDEFRQHLQATGVSLGGVDLTVRVLTTGYWPTQSATPKCNIPPAPRHAFEIFRRFYLAKHSGRQLTLQHHMGSADLNATFYGPVKKEDGSEVGVGGAQVTGSNTRKHILQVSTFQMTILMLFNNREKYTFEEIQQETDIPERELVRALQSLACGKPTQRVLTKEPKSKEIENGHIFTVNDQFTSKLHRVKIQTVAAKQGESDPERKETRQKVDDDRKHEIEAAIVRIMKSRKKMQHNVLVAEVTQQLKARFLPSPVVIKKRIEGLIEREYLARTPEDRKVYTYVA",
# PTC1_HUMAN
"MASAGNAAEPQDRGGGGSGCIGAPGRPAGGGRRRRTGGLRRAAAPDRDYLHRPSYCDAAFALEQISKGKATGRKAPLWLRAKFQRLLFKLGCYIQKNCGKFLVVGLLIFGAFAVGLKAANLETNVEELWVEVGGRVSRELNYTRQKIGEEAMFNPQLMIQTPKEEGANVLTTEALLQHLDSALQASRVHVYMYNRQWKLEHLCYKSGELITETGYMDQIIEYLYPCLIITPLDCFWEGAKLQSGTAYLLGKPPLRWTNFDPLEFLEELKKINYQVDSWEEMLNKAEVGHGYMDRPCLNPADPDCPATAPNKNSTKPLDMALVLNGGCHGLSRKYMHWQEELIVGGTVKNSTGKLVSAHALQTMFQLMTPKQMYEHFKGYEYVSHINWNEDKAAAILEAWQRTYVEVVHQSVAQNSTQKVLSFTTTTLDDILKSFSDVSVIRVASGYLLMLAYACLTMLRWDCSKSQGAVGLAGVLLVALSVAAGLGLCSLIGISFNAATTQVLPFLALGVGVDDVFLLAHAFSETGQNKRIPFEDRTGECLKRTGASVALTSISNVTAFFMAALIPIPALRAFSLQAAVVVVFNFAMVLLIFPAILSMDLYRREDRRLDIFCCFTSPCVSRVIQVEPQAYTDTHDNTRYSPPPPYSSHSFAHETQITMQSTVQLRTEYDPHTHVYYTTAEPRSEISVQPVTVTQDTLSCQSPESTSSTRDLLSQFSDSSLHCLEPPCTKWTLSSFAEKHYAPFLLKPKAKVVVIFLFLGLLGVSLYGTTRVRDGLDLTDIVPRETREYDFIAAQFKYFSFYNMYIVTQKADYPNIQHLLYDLHRSFSNVKYVMLEENKQLPKMWLHYFRDWLQGLQDAFDSDWETGKIMPNNYKNGSDDGVLAYKLLVQTGSRDKPIDISQLTKQRLVDADGIINPSAFYIYLTAWVSNDPVAYAASQANIRPHRPEWVHDKADYMPETRLRIPAAEPIEYAQFPFYLNGLRDTSDFVEAIEKVRTICSNYTSLGLSSYPNGYPFLFWEQYIGLRHWLLLFISVVLACTFLVCAVFLLNPWTAGIIVMVLALMTVELFGMMGLIGIKLSAVPVVILIASVGIGVEFTVHVALAFLTAIGDKNRRAVLALEHMFAPVLDGAVSTLLGVLMLAGSEFDFIVRYFFAVLAILTILGVLNGLVLLPVLLSFFGPYPEVSPANGLNRLPTPSPEPPPSVVRFAMPPGHTHSGSDSSDSEYSSQTTVSGLSEELRHYEAQQGAGGPAHQVIVEATENPVFAHSTVVHPESRHHPPSNPRQQPHLDSGSLPPGRQGQQPRRDPPREGLWPPPYRPRRDAFEISTEGHSGPSNRARWGPRGARSHNPRNPASTAMGSSVPGYCQPITTVTASASVTVAVHPPPVPGPGRNPRGGLCPGYPETDHGLFEDPHVPFHVRCERRDSKVEVIELQDVECEERPRGSSSN",
# 5HT4R_HUMAN
"MDKLDANVSSEEGFGSVEKVVLLTFLSTVILMAILGNLLVMVAVCWDRQLRKIKTNYFIVSLAFADLLVSVLVMPFGAIELVQDIWIYGEVFCLVRTSLDVLLTTASIFHLCCISLDRYYAICCQPLVYRNKMTPLRIALMLGGCWVIPTFISFLPIMQGWNNIGIIDLIEKRKFNQNSNSTYCVFMVNKPYAITCSVVAFYIPFLLMVLAYYRIYVTAKEHAHQIQMLQRAGASSESRPQSADQHSTHRMRTETKAAKTLCIIMGCFCLCWAPFFVTNIVDPFIDYTVPGQVWTAFLWLGYINSGLNPFLYAFLNKSFRRAFLIILCCDDERYRRPSILGQTVPCSTTTINGSTHVLRDAVECGGQWESQCHPPATSPLVAAQPSDT",
# RAPSN_HUMAN
"MGQDQTKQQIEKGLQLYQSNQTEKALQVWTKVLEKSSDLMGRFRVLGCLVTAHSEMGRYKEMLKFAVVQIDTARELEDADFLLESYLNLARSNEKLCEFHKTISYCKTCLGLPGTRAGAQLGGQVSLSMGNAFLGLSVFQKALESFEKALRYAHNNDDAMLECRVCCSLGSFYAQVKDYEKALFFPCKAAELVNNYGKGWSLKYRAMSQYHMAVAYRLLGRLGSAMECCEESMKIALQHGDRPLQALCLLCFADIHRSRGDLETAFPRYDSAMSIMTEIGNRLGQVQALLGVAKCWVARKALDKALDAIERAQDLAEEVGNKLSQLKLHCLSESIYRSKGLQRELRAHVVRFHECVEETELYCGLCGESIGEKNSRLQALPCSHIFHLRCLQNNGTRSCPNCRRSSMKPGFV",
# SPTN1_HUMAN
"MDPSGVKVLETAEDIQERRQQVLDRYHRFKELSTLRRQKLEDSYRFQFFQRDAEELEKWIQEKLQIASDENYKDPTNLQGKLQKHQAFEAEVQANSGAIVKLDETGNLMISEGHFASETIRTRLMELHRQWELLLEKMREKGIKLLQAQKLVQYLRECEDVMDWINDKEAIVTSEELGQDLEHVEVLQKKFEEFQTDMAAHEERVNEVNQFAAKLIQEQHPEEELIKTKQDEVNAAWQRLKGLALQRQGKLFGAAEVQRFNRDVDETISWIKEKEQLMASDDFGRDLASVQALLRKHEGLERDLAALEDKVKALCAEADRLQQSHPLSATQIQVKREELITNWEQIRTLAAERHARLNDSYRLQRFLADFRDLTSWVTEMKALINADELASDVAGAEALLDRHQEHKGEIDAHEDSFKSADESGQALLAAGHYASDEVREKLTVLSEERAALLELWELRRQQYEQCMDLQLFYRDTEQVDNWMSKQEAFLLNEDLGDSLDSVEALLKKHEDFEKSLSAQEEKITALDEFATKLIQNNHYAMEDVATRRDALLSRRNALHERAMRRRAQLADSFHLQQFFRDSDELKSWVNEKMKTATDEAYKDPSNLQGKVQKHQAFEAELSANQSRIDALEKAGQKLIDVNHYAKDEVAARMNEVISLWKKLLEATELKGIKLREANQQQQFNRNVEDIELWLYEVEGHLASDDYGKDLTNVQNLQKKHALLEADVAAHQDRIDGITIQARQFQDAGHFDAENIKKKQEALVARYEALKEPMVARKQKLADSLRLQQLFRDVEDEETWIREKEPIAASTNRGKDLIGVQNLLKKHQALQAEIAGHEPRIKAVTQKGNAMVEEGHFAAEDVKAKLHELNQKWEALKAKASQRRQDLEDSLQAQQYFADANEAESWMREKEPIVGSTDYGKDEDSAEALLKKHEALMSDLSAYGSSIQALREQAQSCRQQVAPTDDETGKELVLALYDYQEKSPREVTMKKGDILTLLNSTNKDWWKVEVNDRQGFVPAAYVKKLDPAQSASRENLLEEQGSIALRQEQIDNQTRITKEAGSVSLRMKQVEELYHSLLELGEKRKGMLEKSCKKFMLFREANELQQWINEKEAALTSEEVGADLEQVEVLQKKFDDFQKDLKANESRLKDINKVAEDLESEGLMAEEVQAVQQQEVYGMMPRDETDSKTASPWKSARLMVHTVATFNSIKELNERWRSLQQLAEERSQLLGSAHEVQRFHRDADETKEWIEEKNQALNTDNYGHDLASVQALQRKHEGFERDLAALGDKVNSLGETAERLIQSHPESAEDLQEKCTELNQAWSSLGKRADQRKAKLGDSHDLQRFLSDFRDLMSWINGIRGLVSSDELAKDVTGAEALLERHQEHRTEIDARAGTFQAFEQFGQQLLAHGHYASPEIKQKLDILDQERADLEKAWVQRRMMLDQCLELQLFHRDCEQAENWMAAREAFLNTEDKGDSLDSVEALIKKHEDFDKAINVQEEKIAALQAFADQLIAAGHYAKGDISSRRNEVLDRWRRLKAQMIEKRSKLGESQTLQQFSRDVDEIEAWISEKLQTASDESYKDPTNIQSKHQKHQAFEAELHANADRIRGVIDMGNSLIERGACAGSEDAVKARLAALADQWQFLVQKSAEKSQKLKEANKQQNFNTGIKDFDFWLSEVEALLASEDYGKDLASVNNLLKKHQLLEADISAHEDRLKDLNSQADSLMTSSAFDTSQVKDKRDTINGRFQKIKSMAASRRAKLNESHRLHQFFRDMDDEESWIKEKKLLVGSEDYGRDLTGVQNLRKKHKRLEAELAAHEPAIQGVLDTGKKLSDDNTIGKEEIQQRLAQFVEHWKELKQLAAARGQRLEESLEYQQFVANVEEEEAWINEKMTLVASEDYGDTLAAIQGLLKKHEAFETDFTVHKDRVNDVCTNGQDLIKKNNHHEENISSKMKGLNGKVSDLEKAAAQRKAKLDENSAFLQFNWKADVVESWIGEKENSLKTDDYGRDLSSVQTLLTKQETFDAGLQAFQQEGIANITALKDQLLAAKHVQSKAIEARHASLMKRWSQLLANSAARKKKLLEAQSHFRKVEDLFLTFAKKASAFNSWFENAEEDLTDPVRCNSLEEIKALREAHDAFRSSLSSAQADFNQLAELDRQIKSFRVASNPYTWFTMEALEETWRNLQKIIKERELELQKEQRRQEENDKLRQEFAQHANAFHQWIQETRTYLLDGSCMVEESGTLESQLEATKRKHQEIRAMRSQLKKIEDLGAAMEEALILDNKYTEHSTVGLAQQWDQLDQLGMRMQHNLEQQIQARNTTGVTEEALKEFSMMFKHFDKDKSGRLNHQEFKSCLRSLGYDLPMVEEGEPDPEFEAILDTVDPNRDGHVSLQEYMAFMISRETENVKSSEEIESAFRALSSEGKPYVTKEELYQNLTREQADYCVSHMKPYVDGKGRELPTAFDYVEFTRSLFVN",
# CAC1C_HUMAN
"MVNENTRMYIPEENHQGSNYGSPRPAHANMNANAAAGLAPEHIPTPGAALSWQAAIDAARQAKLMGSAGNATISTVSSTQRKRQQYGKPKKQGSTTATRPPRALLCLTLKNPIRRACISIVEWKPFEIIILLTIFANCVALAIYIPFPEDDSNATNSNLERVEYLFLIIFTVEAFLKVIAYGLLFHPNAYLRNGWNLLDFIIVVVGLFSAILEQATKADGANALGGKGAGFDVKALRAFRVLRPLRLVSGVPSLQVVLNSIIKAMVPLLHIALLVLFVIIIYAIIGLELFMGKMHKTCYNQEGIADVPAEDDPSPCALETGHGRQCQNGTVCKPGWDGPKHGITNFDNFAFAMLTVFQCITMEGWTDVLYWVNDAVGRDWPWIYFVTLIIIGSFFVLNLVLGVLSGEFSKEREKAKARGDFQKLREKQQLEEDLKGYLDWITQAEDIDPENEDEGMDEEKPRNMSMPTSETESVNTENVAGGDIEGENCGARLAHRISKSKFSRYWRRWNRFCRRKCRAAVKSNVFYWLVIFLVFLNTLTIASEHYNQPNWLTEVQDTANKALLALFTAEMLLKMYSLGLQAYFVSLFNRFDCFVVCGGILETILVETKIMSPLGISVLRCVRLLRIFKITRYWNSLSNLVASLLNSVRSIASLLLLLFLFIIIFSLLGMQLFGGKFNFDEMQTRRSTFDNFPQSLLTVFQILTGEDWNSVMYDGIMAYGGPSFPGMLVCIYFIILFICGNYILLNVFLAIAVDNLADAESLTSAQKEEEEEKERKKLARTASPEKKQELVEKPAVGESKEEKIELKSITADGESPPATKINMDDLQPNENEDKSPYPNPETTGEEDEEEPEMPVGPRPRPLSELHLKEKAVPMPEASAFFIFSSNNRFRLQCHRIVNDTIFTNLILFFILLSSISLAAEDPVQHTSFRNHILFYFDIVFTTIFTIEIALKILGNADYVFTSIFTLEIILKMTAYGAFLHKGSFCRNYFNILDLLVVSVSLISFGIQSSAINVVKILRVLRVLRPLRAINRAKGLKHVVQCVFVAIRTIGNIVIVTTLLQFMFACIGVQLFKGKLYTCSDSSKQTEAECKGNYITYKDGEVDHPIIQPRSWENSKFDFDNVLAAMMALFTVSTFEGWPELLYRSIDSHTEDKGPIYNYRVEISIFFIIYIIIIAFFMMNIFVGFVIVTFQEQGEQEYKNCELDKNQRQCVEYALKARPLRRYIPKNQHQYKVWYVVNSTYFEYLMFVLILLNTICLAMQHYGQSCLFKIAMNILNMLFTGLFTVEMILKLIAFKPKGYFSDPWNVFDFLIVIGSIIDVILSETNHYFCDAWNTFDALIVVGSIVDIAITEVNPAEHTQCSPSMNAEENSRISITFFRLFRVMRLVKLLSRGEGIRTLLWTFIKSFQALPYVALLIVMLFFIYAVIGMQVFGKIALNDTTEINRNNNFQTFPQAVLLLFRCATGEAWQDIMLACMPGKKCAPESEPSNSTEGETPCGSSFAVFYFISFYMLCAFLIINLFVAVIMDNFDYLTRDWSILGPHHLDEFKRIWAEYDPEAKGRIKHLDVVTLLRRIQPPLGFGKLCPHRVACKRLVSMNMPLNSDGTVMFNATLFALVRTALRIKTEGNLEQANEELRAIIKKIWKRTSMKLLDQVVPPAGDDEVTVGKFYATFLIQEYFRKFKKRKEQGLVGKPSQRNALSLQAGLRTLHDIGPEIRRAISGDLTAEEELDKAMKEAVSAASEDDIFRRAGGLFGNHVSYYQSDGRSAFPQTFTTQRPLHINKAGSSQGDTESPSHEKLVDSTFTPSSYSSTGSNANINNANNTALGRLPRPAGYPSTVSTVEGHGPPLSPAIRVQEVAWKLSSNRERHVPMCEDLELRRDSGSAGTQAHCLLLRKANPSRCHSRESQAAMAGQEETSQDETYEVKMNHDTEACSEPSLLSTEMLSYQDDENRQLTLPEEDKRDIRQSPKRGFLRSASLGRRASFHLECLKRQKDRGGDISQKTVLPLHLVHHQALAVAGLSPLLQRSHSPASFPRPFATPPATPGSRGWPPQPVPTLRLEGVESSEKLNSSFPSIHCGSWAETTPGGGGSSAARRVRPVSLMVPSQAGAPGRQFHGSASSLVEAVLISEGLGQFAQDPKFIEVTTQELADACDMTIEEMESAADNILSGGAPQSPNGALLPFVNCRDAGQDRAGGEEDAGCVRARGRPSEEELQDSRVYVSSL",
# RGRF1_HUMAN
"MQKGIRLNDGHVASLGLLARKDGTRKGYLSKRSSDNTKWQTKWFALLQNLLFYFESDSSSRPSGLYLLEGCVCDRAPSPKPALSAKEPLEKQHYFTVNFSHENQKALELRTEDAKDCDEWVAAIAHASYRTLATEHEALMQKYLHLLQIVETEKTVAKQLRQQIEDGEIEIERLKAEITSLLKDNERIQSTQTVAPNDEDSDIKKIKKVQSFLRGWLCRRKWKTIIQDYIRSPHADSMRKRNQVVFSMLEAEAEYVQQLHILVNNFLRPLRMAASSKKPPITHDDVSSIFLNSETIMFLHQIFYQGLKARISSWPTLVLADLFDILLPMLNIYQEFVRNHQYSLQILAHCKQNRDFDKLLKHYEAKPDCEERTLETFLTYPMFQIPRYILTLHELLAHTPHEHVERNSLDYAKSKLEELSRIMHDEVSETENIRKNLAIERMIIEGCEILLDTSQTFVRQGSLIQVPMSEKGKITRGRLGSLSLKKEGERQCFLFSKHLIICTRGSGGKLHLTKNGVISLIDCTLLEEPESTEEEAKGSGQDIDHLDFKIGVEPKDSPPFTVILVASSRQEKAAWTSDISQCVDNIRCNGLMMNAFEENSKVTVPQMIKRTREGTREAEMSRSDASLYCDDVDIRFSKTMNSCKVLQIRYASVERLLERLTDLRFLSIDFLNTFLHSYRVFTTAIVVLDKLITIYKKPISAIPARWLRSLELLFASGQNNKLLYGEPPKSPRATRKFSSPPPLSITKTSSPSRRRKLSLNIPIITGGKALDLAALSCNSNGYTSMYSAMSPFSKATLDTSKLYVSSSFTNKIPDEGDTTPEKPEDPSALSKQSSEVSMREESDIDQNQSDDGDTETSPTKSPTTPKSVKNKNSSEFPLFSYNNGVVMTSCRELDNNRSALSAASAFAIATAGANEGTPNKEKYRRMSLASAGFPPDQRNGDKEFVIRRAATNRVLNVLRHWVSKHSQDFETNDELKCKVIGFLEEVMHDPELLTQERKAAANIIRTLTQEDPGDNQITLEEITQMAEGVKAEPFENHSALEIAEQLTLLDHLVFKKIPYEEFFGQGWMKLEKNERTPYIMKTTKHFNDISNLIASEIIRNEDINARVSAIEKWVAVADICRCLHNYNAVLEITSSMNRSAIFRLKKTWLKVSKQTKALIDKLQKLVSSEGRFKNLREALKNCDPPCVPYLGMYLTDLAFIEEGTPNYTEDGLVNFSKMRMISHIIREIRQFQQTAYKIEHQAKVTQYLLDQSFVMDEESLYESSLRIEPKLPT",
# KCNC3_HUMAN
"MLSSVCVSSFRGRQGASKQQPAPPPQPPESPPPPPLPPQQQQPAQPGPAASPAGPPAPRGPGDRRAEPCPGLPAAAMGRHGGGGGDSGKIVINVGGVRHETYRSTLRTLPGTRLAGLTEPEAAARFDYDPGADEFFFDRHPGVFAYVLNYYRTGKLHCPADVCGPLFEEELGFWGIDETDVEACCWMTYRQHRDAEEALDSFEAPDPAGAANAANAAGAHDGGLDDEAGAGGGGLDGAGGELKRLCFQDAGGGAGGPPGGAGGAGGTWWRRWQPRVWALFEDPYSSRAARYVAFASLFFILISITTFCLETHEGFIHISNKTVTQASPIPGAPPENITNVEVETEPFLTYVEGVCVVWFTFEFLMRITFCPDKVEFLKSSLNIIDCVAILPFYLEVGLSGLSSKAAKDVLGFLRVVRFVRILRIFKLTRHFVGLRVLGHTLRASTNEFLLLIIFLALGVLIFATMIYYAERIGADPDDILGSNHTYFKNIPIGFWWAVVTMTTLGYGDMYPKTWSGMLVGALCALAGVLTIAMPVPVIVNNFGMYYSLAMAKQKLPKKKNKHIPRPPQPGSPNYCKPDPPPPPPPHPHHGSGGISPPPPITPPSMGVTVAGAYPAGPHTHPGLLRGGAGGLGIMGLPPLPAPGEPCPLAQEEVIEINRADPRPNGDPAAAALAHEDCPAIDQPAMSPEDKSPITPGSRGRYSRDRACFLLTDYAPSPDGSIRKATGAPPLPPQDWRKPGPPSFLPDLNANAAAWISP",
# KCC1A_HUMAN
"MLGAVEGPRWKQAEDIRDIYDFRDVLGTGAFSEVILAEDKRTQKLVAIKCIAKEALEGKEGSMENEIAVLHKIKHPNIVALDDIYESGGHLYLIMQLVSGGELFDRIVEKGFYTERDASRLIFQVLDAVKYLHDLGIVHRDLKPENLLYYSLDEDSKIMISDFGLSKMEDPGSVLSTACGTPGYVAPEVLAQKPYSKAVDCWSIGVIAYILLCGYPPFYDENDAKLFEQILKAEYEFDSPYWDDISDSAKDFIRHLMEKDPEKRFTCEQALQHPWIAGDTALDKNIHQSVSEQIKKNFAKSKWKQAFNATAVVRHMRKLQLGTSQEGQGQTASHGELLTPVAGGPAAGCCCRDCCVEPGTELSPTLPHQL",
# HNRPD_HUMAN
"MSEEQFGGDGAAAAATAAVGGSAGEQEGAMVAATQGAAAAAGSGAGTGGGTASGGTEGGSAESEGAKIDASKNEEDEGHSNSSPRHSEAATAQREEWKMFIGGLSWDTTKKDLKDYFSKFGEVVDCTLKLDPITGRSRGFGFVLFKESESVDKVMDQKEHKLNGKVIDPKRAKAMKTKEPVKKIFVGGLSPDTPEEKIREYFGGFGEVESIELPMDNKTNKRRGFCFITFKEEEPVKKIMEKKYHNVGLSKCEIKVAMSKEQYQQQQQWGSRGGFAGRARGRGGGPSQNWNQGYSNYWNQGYGNYGYNSQGYGGYGGYDYTGYNNYYGYGDYSNQQSGYGKVSRRGGHQNSYKPY",
# LRP8_HUMAN
"MGLPEPGPLRLLALLLLLLLLLLLQLQHLAAAAADPLLGGQGPAKDCEKDQFQCRNERCIPSVWRCDEDDDCLDHSDEDDCPKKTCADSDFTCDNGHCIHERWKCDGEEECPDGSDESEATCTKQVCPAEKLSCGPTSHKCVPASWRCDGEKDCEGGADEAGCATLCAPHEFQCGNRSCLAAVFVCDGDDDCGDGSDERGCADPACGPREFRCGGDGGGACIPERWVCDRQFDCEDRSDEAAELCGRPGPGATSAPAACATASQFACRSGECVHLGWRCDGDRDCKDKSDEADCPLGTCRGDEFQCGDGTCVLAIKHCNQEQDCPDGSDEAGCLQGLNECLHNNGGCSHICTDLKIGFECTCPAGFQLLDQKTCGDIDECKDPDACSQICVNYKGYFKCECYPGYEMDLLTKNCKAAAGKSPSLIFTNRHEVRRIDLVKRNYSRLIPMLKNVVALDVEVATNRIYWCDLSYRKIYSAYMDKASDPKEQEVLIDEQLHSPEGLAVDWVHKHIYWTDSGNKTISVATVDGGRRRTLFSRNLSEPRAIAVDPLRGFMYWSDWGDQAKIEKSGLNGVDRQTLVSDNIEWPNGITLDLLSQRLYWVDSKLHQLSSIDFSGGNRKTLISSTDFLSHPFGIAVFEDKVFWTDLENEAIFSANRLNGLEISILAENLNNPHDIVIFHELKQPRAPDACELSVQPNGGCEYLCLPAPQISSHSPKYTCACPDTMWLGPDMKRCYRAPQSTSTTTLASTMTRTVPATTRAPGTTVHRSTYQNHSTETPSLTAAVPSSVSVPRAPSISPSTLSPATSNHSQHYANEDSKMGSTVTAAVIGIIVPIVVIALLCMSGYLIWRNWKRKNTKSMNFDNPVYRKTTEEEDEDELHIGRTAQIGHVYPAAISSFDRPLWAEPCLGETREPEDPAPALKELFVLPGEPRSQLHQLPKNPLSELPVVKSKRVALSLEDDGLP",
# DAG1_HUMAN
"MRMSVGLSLLLPLSGRTFLLLLSVVMAQSHWPSEPSEAVRDWENQLEASMHSVLSDLHEAVPTVVGIPDGTAVVGRSFRVTIPTDLIASSGDIIKVSAAGKEALPSWLHWDSQSHTLEGLPLDTDKGVHYISVSATRLGANGSHIPQTSSVFSIEVYPEDHSELQSVRTASPDPGEVVSSACAADEPVTVLTVILDADLTKMTPKQRIDLLHRMRSFSEVELHNMKLVPVVNNRLFDMSAFMAGPGNAKKVVENGALLSWKLGCSLNQNSVPDIHGVEAPAREGAMSAQLGYPVVGWHIANKKPPLPKRVRRQIHATPTPVTAIGPPTTAIQEPPSRIVPTPTSPAIAPPTETMAPPVRDPVPGKPTVTIRTRGAIIQTPTLGPIQPTRVSEAGTTVPGQIRPTMTIPGYVEPTAVATPPTTTTKKPRVSTPKPATPSTDSTTTTTRRPTKKPRTPRPVPRVTTKVSITRLETASPPTRIRTTTSGVPRGGEPNQRPELKNHIDRVDAWVGTYFEVKIPSDTFYDHEDTTTDKLKLTLKLREQQLVGEKSWVQFNSNSQLMYGLPDSSHVGKHEYFMHATDKGGLSAVDAFEIHVHRRPQGDRAPARFKAKFVGDPALVLNDIHKKIALVKKLAFAFGDRNCSTITLQNITRGSIVVEWTNNTLPLEPCPKEQIAGLSRRIAEDDGKPRPAFSNALEPDFKATSITVTGSGSCRHLQFIPVVPPRRVPSEAPPTEVPDRDPEKSSEDDVYLHTVIPAVVVAAILLIAGIIAMICYRKKRKGKLTLEDQATFIKKGVPIIFADELDDSKPPPSSSMPLILQEEKAPLPPPEYPNQSVPETTPLNQDTMGEYTPLRDEDPNAPPYQPPPPFTAPMEGKGSRPKNMTPYRSPPPYVPP",
# SEPT6_HUMAN
"MAATDIARQVGEGCRTVPLAGHVGFDSLPDQLVNKSVSQGFCFNILCVGETGLGKSTLMDTLFNTKFEGEPATHTQPGVQLQSNTYDLQESNVRLKLTIVSTVGFGDQINKEDSYKPIVEFIDAQFEAYLQEELKIRRVLHTYHDSRIHVCLYFIAPTGHSLKSLDLVTMKKLDSKVNIIPIIAKADAISKSELTKFKIKITSELVSNGVQIYQFPTDDESVAEINGTMNAHLPFAVIGSTEELKIGNKMMRARQYPWGTVQVENEAHCDFVKLREMLIRVNMEDLREQTHTRHYELYRRCKLEEMGFKDTDPDSKPFSLQETYEAKRNEFLGELQKKEEEMRQMFVQRVKEKEAELKEAEKELHEKFDRLKKLHQDEKKKLEDKKKSLDDEVNAFKQRKTAAELLQSQGSQAGGSQTLKRDKEKKNNPWLCTE",
# EIF3A_HUMAN
"MPAYFQRPENALKRANEFLEVGKKQPALDVLYDVMKSKKHRTWQKIHEPIMLKYLELCVDLRKSHLAKEGLYQYKNICQQVNIKSLEDVVRAYLKMAEEKTEAAKEESQQMVLDIEDLDNIQTPESVLLSAVSGEDTQDRTDRLLLTPWVKFLWESYRQCLDLLRNNSRVERLYHDIAQQAFKFCLQYTRKAEFRKLCDNLRMHLSQIQRHHNQSTAINLNNPESQSMHLETRLVQLDSAISMELWQEAFKAVEDIHGLFSLSKKPPKPQLMANYYNKVSTVFWKSGNALFHASTLHRLYHLSREMRKNLTQDEMQRMSTRVLLATLSIPITPERTDIARLLDMDGIIVEKQRRLATLLGLQAPPTRIGLINDMVRFNVLQYVVPEVKDLYNWLEVEFNPLKLCERVTKVLNWVREQPEKEPELQQYVPQLQNNTILRLLQQVSQIYQSIEFSRLTSLVPFVDAFQLERAIVDAARHCDLQVRIDHTSRTLSFGSDLNYATREDAPIGPHLQSMPSEQIRNQLTAMSSVLAKALEVIKPAHILQEKEEQHQLAVTAYLKNSRKEHQRILARRQTIEERKERLESLNIQREKEELEQREAELQKVRKAEEERLRQEAKEREKERILQEHEQIKKKTVRERLEQIKKTELGAKAFKDIDIEDLEELDPDFIMAKQVEQLEKEKKELQERLKNQEKKIDYFERAKRLEEIPLIKSAYEEQRIKDMDLWEQQEEERITTMQLEREKALEHKNRMSRMLEDRDLFVMRLKAARQSVYEEKLKQFEERLAEERHNRLEERKRQRKEERRITYYREKEEEEQRRAEEQMLKEREERERAERAKREEELREYQERVKKLEEVERKKRQRELEIEERERRREEERRLGDSSLSRKDSRWGDRDSEGTWRKGPEADSEWRRGPPEKEWRRGEGRDEDRSHRRDEERPRRLGDDEDREPSLRPDDDRVPRRGMDDDRGPRRGPEEDRFSRRGADDDRPSWRNTDDDRPPRRIADEDRGNWRHADDDRPPRRGLDEDRGSWRTADEDRGPRRGMDDDRGPRRGGADDERSSWRNADDDRGPRRGLDDDRGPRRGMDDDRGPRRGMDDDRGPRRGMDDDRGPRRGLDDDRGPWRNADDDRIPRRGAEDDRGPWRNMDDDRLSRRADDDRFPRRGDDSRPGPWRPLVKPGGWREKEKAREESWGPPRESRPSEEREWDREKERDRDNQDREENDKDPERERDRERDVDREDRFRRPRDEGGWRRGPAEESSSWRDSSRRDDRDRDDRRRERDDRRDLRERRDLRDDRDRRGPPLRSEREEVSSWRRADDRKDDRVEERDPPRRVPPPALSRDRERDRDREREGEKEKASWRAEKDRESLRRTKNETDEDGWTTVRR",
# ARHG7_HUMAN
"MNSAEQTVTWLITLGVLESPKKTISDPEGFLQASLKDGVVLCRLLERLLPGTIEKVYPEPRSESECLSNIREFLRGCGASLRLELLFPPSQPPQHLVTTILLSASTFDANDLYQGQNFNKVLSSLVTLNKVTADIGLGSDSVCARPSSHRIKSFDSLGSQSLHTRTSKLFQGQYRSLDMTDNSNNQLVVRAKFNFQQTNEDELSFSKGDVIHVTRVEEGGWWEGTLNGRTGWFPSNYVREVKASEKPVSPKSGTLKSPPKGFDTTAINKSYYNVVLQNILETENEYSKELQTVLSTYLRPLQTSEKLSSANISYLMGNLEEICSFQQMLVQSLEECTKLPEAQQRVGGCFLNLMPQMKTLYLTYCANHPSAVNVLTEHSEELGEFMETKGASSPGILVLTTGLSKPFMRLDKYPTLLKELERHMEDYHTDRQDIQKSMAAFKNLSAQCQEVRKRKELELQILTEAIRNWEGDDIKTLGNVTYMSQVLIQCAGSEEKNERYLLLFPNVLLMLSASPRMSGFIYQGKLPTTGMTITKLEDSENHRNAFEISGSMIERILVSCNNQQDLQEWVEHLQKQTKVTSVGNPTIKPHSVPSHTLPSHPVTPSSKHADSKPAPLTPAYHTLPHPSHHGTPHTTINWGPLEPPKTPKPWSLSCLRPAPPLRPSAALCYKEDLSKSPKTMKKLLPKRKPERKPSDEEFASRKSTAALEEDAQILKVIEAYCTSAKTRQTLNSTWQGTDLMHNHVLADDDQPSLDSLGRRSSLSRLEPSDLSEDSDYDSIWTAHSYRMGSTSRKSCCSYISHQN",
# EFR3A_HUMAN
"MPTRVCCCCSALRPRYKRLVDNIFPEDPKDGLVKTDMEKLTFYAVSAPEKLDRIGSYLAERLSRDVVRHRSGYVLIAMEALDQLLMACHSQSIKPFVESFLHMVAKLLESGEPKLQVLGTNSFVKFANIEEDTPSYHRRYDFFVSRFSAMCHSCHSDPEIRTEIRIAGIRGIQGVVRKTVNDELRATIWEPQHMDKIVPSLLFNMQKIEEVDSRIGPPSSPSATDKEENPAVLAENCFRELLGRATFGNMNNAVRPVFAHLDHHKLWDPNEFAVHCFKIIMYSIQAQYSHHVIQEILGHLDARKKDAPRVRAGIIQVLLEAVAIAAKGSIGPTVLEVFNTLLKHLRLSVEFEANDLQGGSVGSVNLNTSSKDNDEKIVQNAIIQTIGFFGSNLPDYQRSEIMMFIMGKVPVFGTSTHTLDISQLGDLGTRRIQIMLLRSLLMVTSGYKAKTIVTALPGSFLDPLLSPSLMEDYELRQLVLEVMHNLMDRHDNRAKLRGIRIIPDVADLKIKREKICRQDTSFMKKNGQQLYRHIYLGCKEEDNVQKNYELLYTSLALITIELANEEVVIDLIRLAIALQDSAIINEDNLPMFHRCGIMALVAAYLNFVSQMIAVPAFCQHVSKVIEIRTMEAPYFLPEHIFRDKCMLPKSLEKHEKDLYFLTNKIAESLGGSGYSVERLSVPYVPQVTDEDRLSRRKSIVDTVSIQVDILSNNVPSDDVVSNTEEITFEALKKAIDTSGMEEQEKEKRRLVIEKFQKAPFEEIAAQCESKANLLHDRLAQILELTIRPPPSPSGTLTITSGHAQYQSVPVYEMKFPDLCVY",
# SCRIB_HUMAN
"MLKCIPLWRCNRHVESVDKRHCSLQAVPEEIYRYSRSLEELLLDANQLRELPKPFFRLLNLRKLGLSDNEIQRLPPEVANFMQLVELDVSRNDIPEIPESIKFCKALEIADFSGNPLSRLPDGFTQLRSLAHLALNDVSLQALPGDVGNLANLVTLELRENLLKSLPASLSFLVKLEQLDLGGNDLEVLPDTLGALPNLRELWLDRNQLSALPPELGNLRRLVCLDVSENRLEELPAELGGLVLLTDLLLSQNLLRRLPDGIGQLKQLSILKVDQNRLCEVTEAIGDCENLSELILTENLLMALPRSLGKLTKLTNLNVDRNHLEALPPEIGGCVALSVLSLRDNRLAVLPPELAHTTELHVLDVAGNRLQSLPFALTHLNLKALWLAENQAQPMLRFQTEDDARTGEKVLTCYLLPQQPPPSLEDAGQQGSLSETWSDAPPSRVSVIQFLEAPIGDEDAEEAAAEKRGLQRRATPHPSELKVMKRSIEGRRSEACPCQPDSGSPLPAEEEKRLSAESGLSEDSRPSASTVSEAEPEGPSAEAQGGSQQEATTAGGEEDAEEDYQEPTVHFAEDALLPGDDREIEEGQPEAPWTLPGGRQRLIRKDTPHYKKHFKISKLPQPEAVVALLQGMQPDGEGPVAPGGWHNGPHAPWAPRAQKEEEEEEEGSPQEEEEEEEEENRAEEEEASTEEEDKEGAVVSAPSVKGVSFDQANNLLIEPARIEEEELTLTILRQTGGLGISIAGGKGSTPYKGDDEGIFISRVSEEGPAARAGVRVGDKLLEVNGVALQGAEHHEAVEALRGAGTAVQMRVWRERMVEPENAVTITPLRPEDDYSPRERRGGGLRLPLLPPESPGPLRQRHVACLARSERGLGFSIAGGKGSTPYRAGDAGIFVSRIAEGGAAHRAGTLQVGDRVLSINGVDVTEARHDHAVSLLTAASPTIALLLEREAGGPLPPSPLPHSSPPTAAVATTSITTATPGVPGLPSLAPSLLAAALEGPYPVEEIRLPRAGGPLGLSIVGGSDHSSHPFGVQEPGVFISKVLPRGLAARSGLRVGDRILAVNGQDVRDATHQEAVSALLRPCLELSLLVRRDPAPPGLRELCIQKAPGERLGISIRGGARGHAGNPRDPTDEGIFISKVSPTGAAGRDGRLRVGLRLLEVNQQSLLGLTHGEAVQLLRSVGDTLTVLVCDGFEASTDAALEVSPGVIANPFAAGIGHRNSLESISSIDRELSPEGPGKEKELPGQTLHWGPEATEAAGRGLQPLKLDYRALAAVPSAGSVQRVPSGAAGGKMAESPCSPSGQQPPSPPSPDELPANVKQAYRAFAAVPTSHPPEDAPAQPPTPGPAASPEQLSFRERQKYFELEVRVPQAEGPPKRVSLVGADDLRKMQEEEARKLQQKRAQMLREAAEAGAEARLALDGETLGEEEQEDEQPPWASPSPTSRQSPASPPPLGGGAPVRTAKAERRHQERLRVQSPEPPAPERALSPAELRALEAEKRALWRAARMKSLEQDALRAQMVLSRSQEGRGTRGPLERLAEAPSPAPTPSPTPVEDLGPQTSTSPGRLPLSGKKFDYRAFAALPSSRPVYDIQSPDFAEELRSLEPSPSPGPQEEDGEVALVLLGRPSPGAVGPEDVALCSSRRPVRPGRRGLGPVPS",
# GIT2_HUMAN
"MSKRLRSSEVCADCSGPDPSWASVNRGTFLCDECCSVHRSLGRHISQVRHLKHTPWPPTLLQMVETLYNNGANSIWEHSLLDPASIMSGRRKANPQDKVHPNKAEFIRAKYQMLAFVHRLPCRDDDSVTAKDLSKQLHSSVRTGNLETCLRLLSLGAQANFFHPEKGNTPLHVASKAGQILQAELLAVYGADPGTQDSSGKTPVDYARQGGHHELAERLVEIQYELTDRLAFYLCGRKPDHKNGQHFIIPQMADSSLDLSELAKAAKKKLQSLSNHLFEELAMDVYDEVDRRETDAVWLATQNHSALVTETTVVPFLPVNPEYSSTRNQGRQKLARFNAHEFATLVIDILSDAKRRQQGSSLSGSKDNVELILKTINNQHSVESQDNDQPDYDSVASDEDTDLETTASKTNRQKSLDSDLSDGPVTVQEFMEVKNALVASEAKIQQLMKVNNNLSDELRIMQKKLQTLQSENSNLRKQATTNVYQVQTGSEYTDTSNHSSLKRRPSARGSRPMSMYETGSGQKPYLPMGEASRPEESRMRLQPFPAHIGRSALVTSSSSLPSFPSTLSWSRDESARRASRLEKQNSTPESDYDNTPNDMEPDGMGSSRKGRQRSMVWPGDGLVPDTAEPHVAPSPTLPSTEDVIRKTEQITKNIQELLRAAQENKHDSYIPCSERIHVAVTEMAALFPKKPKSDMVRTSLRLLTSSAYRLQSECKKTLPGDPGSPTDVQLVTQQVIQCAYDIAKAAKQLVTITTKENNN",
# MPP2_HUMAN
"MPVAATNSETAMQQVLDNLGSLPSATGAAELDLIFLRGIMESPIVRSLAKVIMVLWFMQQNVFVPMKYMLKYFGAHERLEETKLEAVRDNNLELVQEILRDLAHVAEQSSTAAELAHILQEPHFQSLLETHDSVASKTYETPPPSPGLDPTFSNQPVPPDAVRMVGIRKTAGEHLGVTFRVEGGELVIARILHGGMVAQQGLLHVGDIIKEVNGQPVGSDPRALQELLRNASGSVILKILPSYQEPHLPRQVFVKCHFDYDPARDSLIPCKEAGLRFNAGDLLQIVNQDDANWWQACHVEGGSAGLIPSQLLEEKRKAFVKRDLELTPNSGTLCGSLSGKKKKRMMYLTTKNAEFDRHELLIYEEVARMPPFRRKTLVLIGAQGVGRRSLKNKLIMWDPDRYGTTVPYTSRRPKDSEREGQGYSFVSRGEMEADVRAGRYLEHGEYEGNLYGTRIDSIRGVVAAGKVCVLDVNPQAVKVLRTAEFVPYVVFIEAPDFETLRAMNRAALESGISTKQLTEADLRRTVEESSRIQRGYGHYFDLCLVNSNLERTFRELQTAMEKLRTEPQWVPVSWVY",
# DOC2A_HUMAN
"MRGRRGDRMTINIQEHMAINVCPGPIRPIRQISDYFPRGPGPEGGGGGGGEAPAHLVPLALAPPAALLGATTPEDGAEVDSYDSDDATALGTLEFDLLYDRASCTLHCSILRAKGLKPMDFNGLADPYVKLHLLPGACKANKLKTKTQRNTLNPVWNEDLTYSGITDDDITHKVLRIAVCDEDKLSHNEFIGEIRVPLRRLKPSQKKHFNICLERQVPLASPSSMSAALRGISCYLKELEQAEQGQGLLEERGRILLSLSYSSRRRGLLVGILRCAHLAAMDVNGYSDPYVKTYLRPDVDKKSKHKTCVKKKTLNPEFNEEFFYEIELSTLATKTLEVTVWDYDIGKSNDFIGGVSLGPGARGEARKHWSDCLQQPDAALERWHTLTSELPPAAGALSSA",
# DOC2B_HUMAN
"MTLRRRGEKATISIQEHMAIDVCPGPIRPIKQISDYFPRFPRGLPPDAGPRAAAPPDAPARPAVAGAGRRSPSDGAREDDEDVDQLFGAYGSSPGPSPGPSPARPPAKPPEDEPDADGYESDDCTALGTLDFSLLYDQENNALHCTITKAKGLKPMDHNGLADPYVKLHLLPGASKANKLRTKTLRNTLNPTWNETLTYYGITDEDMIRKTLRISVCDEDKFRHNEFIGETRVPLKKLKPNHTKTFSICLEKQLPVDKTEDKSLEERGRILISLKYSSQKQGLLVGIVRCAHLAAMDANGYSDPYVKTYLRPDVDKKSKHKTAVKKKTLNPEFNEEFCYEIKHGDLAKKSLEVTVWDYDIGKSNDFIGGVVLGIHAKGERLKHWFDCLKNKDKRIERWHTLTSELPGAVLSD",
# DOCK1_HUMAN
"MTRWVPTKREEKYGVAFYNYDARGADELSLQIGDTVHILETYEGWYRGYTLRKKSKKGIFPASYIHLKEAIVEGKGQHETVIPGDLPLIQEVTTTLREWSTIWRQLYVQDNREMFRSVRHMIYDLIEWRSQILSGTLPQDELKELKKKVTAKIDYGNRILDLDLVVRDEDGNILDPELTSTISLFRAHEIASKQVEERLQEEKSQKQNIDINRQAKFAATPSLALFVNLKNVVCKIGEDAEVLMSLYDPVESKFISENYLVRWSSSGLPKDIDRLHNLRAVFTDLGSKDLKREKISFVCQIVRVGRMELRDNNTRKLTSGLRRPFGVAVMDVTDIINGKVDDEDKQHFIPFQPVAGENDFLQTVINKVIAAKEVNHKGQGLWVTLKLLPGDIHQIRKEFPHLVDRTTAVARKTGFPEIIMPGDVRNDIYVTLVQGDFDKGSKTTAKNVEVTVSVYDEDGKRLEHVIFPGAGDEAISEYKSVIYYQVKQPRWFETVKVAIPIEDVNRSHLRFTFRHRSSQDSKDKSEKIFALAFVKLMRYDGTTLRDGEHDLIVYKAEAKKLEDAATYLSLPSTKAELEEKGHSATGKSMQSLGSCTISKDSFQISTLVCSTKLTQNVDLLGLLKWRSNTSLLQQNLRQLMKVDGGEVVKFLQDTLDALFNIMMENSESETFDTLVFDALVFIIGLIADRKFQHFNPVLETYIKKHFSATLAYTKLTKVLKNYVDGAEKPGVNEQLYKAMKALESIFKFIVRSRILFNQLYENKGEADFVESLLQLFRSINDMMSSMSDQTVRVKGAALKYLPTIVNDVKLVFDPKELSKMFTEFILNVPMGLLTIQKLYCLIEIVHSDLFTQHDCREILLPMMTDQLKYHLERQEDLEACCQLLSHILEVLYRKDVGPTQRHVQIIMEKLLRTVNRTVISMGRDSELIGNFVACMTAILRQMEDYHYAHLIKTFGKMRTDVVDFLMETFIMFKNLIGKNVYPFDWVIMNMVQNKVFLRAINQYADMLNKKFLDQANFELQLWNNYFHLAVAFLTQESLQLENFSSAKRAKILNKYGDMRRQIGFEIRDMWYNLGQHKIKFIPEMVGPILEMTLIPETELRKATIPIFFDMMQCEFHSTRSFQMFENEIITKLDHEVEGGRGDEQYKVLFDKILLEHCRKHKYLAKTGETFVKLVVRLMERLLDYRTIMHDENKENRMSCTVNVLNFYKEIEREEMYIRYLYKLCDLHKECDNYTEAAYTLLLHAKLLKWSEDVCVAHLTQRDGYQATTQGQLKEQLYQEIIHYFDKGKMWEEAIALGKELAEQYENEMFDYEQLSELLKKQAQFYENIVKVIRPKPDYFAVGYYGQGFPTFLRGKVFIYRGKEYERREDFEARLLTQFPNAEKMKTTSPPGDDIKNSPGQYIQCFTVKPKLDLPPKFHRPVSEQIVSFYRVNEVQRFEYSRPIRKGEKNPDNEFANMWIERTIYTTAYKLPGILRWFEVKSVFMVEISPLENAIETMQLTNDKINSMVQQHLDDPSLPINPLSMLLNGIVDPAVMGGFANYEKAFFTDRYLQEHPEAHEKIEKLKDLIAWQIPFLAEGIRIHGDKVTEALRPFHERMEACFKQLKEKVEKEYGVRIMPSSLDDRRGSRPRSMVRSFTMPSSSRPLSVASVSSLSSDSTPSRPGSDGFALEPLLPKKMHSRSQDKLDKDDLEKEKKDKKKEKRNSKHQEIFEKEFKPTDISLQQSEAVILSETISPLRPQRPKSQVMNVIGSERRFSVSPSSPSSQQTPPPVTPRAKLSFSMQSSLELNGMTGADVADVPPPLPLKGSVADYGNLMENQDLLGSPTPPPPPPHQRHLPPPLPSKTPPPPPPKTTRKQASVDSGIVQ",
# DPYL1_HUMAN
"MSYQGKKSIPHITSDRLLIKGGRIINDDQSLYADVYLEDGLIKQIGENLIVPGGVKTIEANGRMVIPGGIDVNTYLQKPSQGMTAADDFFQGTRAALVGGTTMIIDHVVPEPGSSLLTSFEKWHEAADTKSCCDYSLHVDITSWYDGVREELEVLVQDKGVNSFQVYMAYKDVYQMSDSQLYEAFTFLKGLGAVILVHAENGDLIAQEQKRILEMGITGPEGHALSRPEELEAEAVFRAITIAGRINCPVYITKVMSKSAADIIALARKKGPLVFGEPIAASLGTDGTHYWSKNWAKAAAFVTSPPLSPDPTTPDYLTSLLACGDLQVTGSGHCPYSTAQKAVGKDNFTLIPEGVNGIEERMTVVWDKAVATGKMDENQFVAVTSTNAAKIFNLYPRKGRIAVGSDADVVIWDPDKLKTITAKSHKSAVEYNIFEGMECHGSPLVVISQGKIVFEDGNINVNKGMGRFIPRKAFPEHLYQRVKIRNKVFGLQGVSRGMYDGPVYEVPATPKYATPAPSAKSSPSKHQPPPIRNLHQSNFSLSGAQIDDNNPRRTGHRIVAPPGGRSNITSLG",
# DPYL3_HUMAN
"MSYQGKKNIPRITSDRLLIKGGRIVNDDQSFYADIYMEDGLIKQIGDNLIVPGGVKTIEANGKMVIPGGIDVHTHFQMPYKGMTTVDDFFQGTKAALAGGTTMIIDHVVPEPESSLTEAYEKWREWADGKSCCDYALHVDITHWNDSVKQEVQNLIKDKGVNSFMVYMAYKDLYQVSNTELYEIFTCLGELGAIAQVHAENGDIIAQEQTRMLEMGITGPEGHVLSRPEELEAEAVFRAITIASQTNCPLYVTKVMSKSAADLISQARKKGNVVFGEPITASLGIDGTHYWSKNWAKAAAFVTSPPLSPDPTTPDYINSLLASGDLQLSGSAHCTFSTAQKAIGKDNFTAIPEGTNGVEERMSVIWDKAVATGKMDENQFVAVTSTNAAKIFNLYPRKGRISVGSDSDLVIWDPDAVKIVSAKNHQSAAEYNIFEGMELRGAPLVVICQGKIMLEDGNLHVTQGAGRFIPCSPFSDYVYKRIKARRKMADLHAVPRGMYDGPVFDLTTTPKGGTPAGSARGSPTRPNPPVRNLHQSGFSLSGTQVDEGVRSASKRIVAPPGGRSNITSLS",
# SRC8_HUMAN
"MWKASAGHAVSIAQDDAGADDWETDPDFVNDVSEKEQRWGAKTVQGSGHQEHINIHKLRENVFQEHQTLKEKELETGPKASHGYGGKFGVEQDRMDKSAVGHEYQSKLSKHCSQVDSVRGFGGKFGVQMDRVDQSAVGFEYQGKTEKHASQKDYSSGFGGKYGVQADRVDKSAVGFDYQGKTEKHESQRDYSKGFGGKYGIDKDKVDKSAVGFEYQGKTEKHESQKDYVKGFGGKFGVQTDRQDKCALGWDHQEKLQLHESQKDYKTGFGGKFGVQSERQDSAAVGFDYKEKLAKHESQQDYSKGFGGKYGVQKDRMDKNASTFEDVTQVSSAYQKTVPVEAVTSKTSNIRANFENLAKEKEQEDRRKAEAERAQRMAKERQEQEEARRKLEEQARAKTQTPPVSPAPQPTEERLPSSPVYEDAASFKAELSYRGPVSGTEPEPVYSMEAADYREASSQQGLAYATEAVYESAEAPGHYPAEDSTYDEYENDLGITAVALYDYQAAGDDEISFDPDDIITNIEMIDDGWWRGVCKGRYGLFPANYVELRQ",
# FLOT2_HUMAN
"MGNCHTVGPNEALVVSGGCCGSDYKQYVFGGWAWAWWCISDTQRISLEIMTLQPRCEDVETAEGVALTVTGVAQVKIMTEKELLAVACEQFLGKNVQDIKNVVLQTLEGHLRSILGTLTVEQIYQDRDQFAKLVREVAAPDVGRMGIEILSFTIKDVYDKVDYLSSLGKTQTAVVQRDADIGVAEAERDAGIREAECKKEMLDVKFMADTKIADSKRAFELQKSAFSEEVNIKTAEAQLAYELQGAREQQKIRQEEIEIEVVQRKKQIAVEAQEILRTDKELIATVRRPAEAEAHRIQQIAEGEKVKQVLLAQAEAEKIRKIGEAEAAVIEAMGKAEAERMKLKAEAYQKYGDAAKMALVLEALPQIAAKIAAPLTKVDEIVVLSGDNSKVTSEVNRLLAELPASVHALTGVDLSKIPLIKKATGVQV",
# FAK2_HUMAN
"MSGVSEPLSRVKLGTLRRPEGPAEPMVVVPVDVEKEDVRILKVCFYSNSFNPGKNFKLVKCTVQTEIREIITSILLSGRIGPNIRLAECYGLRLKHMKSDEIHWLHPQMTVGEVQDKYECLHVEAEWRYDLQIRYLPEDFMESLKEDRTTLLYFYQQLRNDYMQRYASKVSEGMALQLGCLELRRFFKDMPHNALDKKSNFELLEKEVGLDLFFPKQMQENLKPKQFRKMIQQTFQQYASLREEECVMKFFNTLAGFANIDQETYRCELIQGWNITVDLVIGPKGIRQLTSQDAKPTCLAEFKQIRSIRCLPLEEGQAVLQLGIEGAPQALSIKTSSLAEAENMADLIDGYCRLQGEHQGSLIIHPRKDGEKRNSLPQIPMLNLEARRSHLSESCSIESDIYAEIPDETLRRPGGPQYGIAREDVVLNRILGEGFFGEVYEGVYTNHKGEKINVAVKTCKKDCTLDNKEKFMSEAVIMKNLDHPHIVKLIGIIEEEPTWIIMELYPYGELGHYLERNKNSLKVLTLVLYSLQICKAMAYLESINCVHRDIAVRNILVASPECVKLGDFGLSRYIEDEDYYKASVTRLPIKWMSPESINFRRFTTASDVWMFAVCMWEILSFGKQPFFWLENKDVIGVLEKGDRLPKPDLCPPVLYTLMTRCWDYDPSDRPRFTELVCSLSDVYQMEKDIAMEQERNARYRTPKILEPTAFQEPPPKPSRPKYRPPPQTNLLAPKLQFQVPEGLCASSPTLTSPMEYPSPVNSLHTPPLHRHNVFKRHSMREEDFIQPSSREEAQQLWEAEKVKMRQILDKQQKQMVEDYQWLRQEEKSLDPMVYMNDKSPLTPEKEVGYLEFTGPPQKPPRLGAQSIQPTANLDRTDDLVYLNVMELVRAVLELKNELCQLPPEGYVVVVKNVGLTLRKLIGSVDDLLPSLPSSSRTEIEGTQKLLNKDLAELINKMRLAQQNAVTSLSEECKRQMLTASHTLAVDAKNLLDAVDQAKVLANLAHPPAE",
# GNA13_HUMAN
"MADFLPSRSVLSVCFPGCLLTSGEAEQQRKSKEIDKCLSREKTYVKRLVKILLLGAGESGKSTFLKQMRIIHGQDFDQRAREEFRPTIYSNVIKGMRVLVDAREKLHIPWGDNSNQQHGDKMMSFDTRAPMAAQGMVETRVFLQYLPAIRALWADSGIQNAYDRRREFQLGESVKYFLDNLDKLGEPDYIPSQQDILLARRPTKGIHEYDFEIKNVPFKMVDVGGQRSERKRWFECFDSVTSILFLVSSSEFDQVLMEDRLTNRLTESLNIFETIVNNRVFSNVSIILFLNKTDLLEEKVQIVSIKDYFLEFEGDPHCLRDVQKFLVECFRNKRRDQQQKPLYHHFTTAINTENIRLVFRDVKDTILHDNLKQLMLQ",
# GRM2_HUMAN
"MGSLLALLALLLLWGAVAEGPAKKVLTLEGDLVLGGLFPVHQKGGPAEDCGPVNEHRGIQRLEAMLFALDRINRDPHLLPGVRLGAHILDSCSKDTHALEQALDFVRASLSRGADGSRHICPDGSYATHGDAPTAITGVIGGSYSDVSIQVANLLRLFQIPQISYASTSAKLSDKSRYDYFARTVPPDFFQAKAMAEILRFFNWTYVSTVASEGDYGETGIEAFELEARARNICVATSEKVGRAMSRAAFEGVVRALLQKPSARVAVLFTRSEDARELLAASQRLNASFTWVASDGWGALESVVAGSEGAAEGAITIELASYPISDFASYFQSLDPWNNSRNPWFREFWEQRFRCSFRQRDCAAHSLRAVPFEQESKIMFVVNAVYAMAHALHNMHRALCPNTTRLCDAMRPVNGRRLYKDFVLNVKFDAPFRPADTHNEVRFDRFGDGIGRYNIFTYLRAGSGRYRYQKVGYWAEGLTLDTSLIPWASPSAGPLPASRCSEPCLQNEVKSVQPGEVCCWLCIPCQPYEYRLDEFTCADCGLGYWPNASLTGCFELPQEYIRWGDAWAVGPVTIACLGALATLFVLGVFVRHNATPVVKASGRELCYILLGGVFLCYCMTFIFIAKPSTAVCTLRRLGLGTAFSVCYSALLTKTNRIARIFGGAREGAQRPRFISPASQVAICLALISGQLLIVVAWLVVEAPGTGKETAPERREVVTLRCNHRDASMLGSLAYNVLLIALCTLYAFKTRKCPENFNEAKFIGFTMYTTCIIWLAFLPIFYVTSSDYRVQTTTMCVSVSLSGSVVLGCLFAPKLHIILFQPQKNVVSHRAPTSRFGSAAARASSSLGQGSGSQFVPTVCNGREVVDSTTSSL",
# CAPR1_HUMAN
"MPSATSHSGSGSKSSGPPPPSGSSGSEAAAGAGAAAPASQHPATGTGAVQTEAMKQILGVIDKKLRNLEKKKGKLDDYQERMNKGERLNQDQLDAVSKYQEVTNNLEFAKELQRSFMALSQDIQKTIKKTARREQLMREEAEQKRLKTVLELQYVLDKLGDDEVRTDLKQGLNGVPILSEEELSLLDEFYKLVDPERDMSLRLNEQYEHASIHLWDLLEGKEKPVCGTTYKVLKEIVERVFQSNYFDSTHNHQNGLCEEEEAASAPAVEDQVPEAEPEPAEEYTEQSEVESTEYVNRQFMAETQFTSGEKEQVDEWTVETVEVVNSLQQQPQAASPSVPEPHSLTPVAQADPLVRRQRVQDLMAQMQGPYNFIQDSMLDFENQTLDPAIVSAQPMNPTQNMDMPQLVCPPVHSESRLAQPNQVPVQPEATQVPLVSSTSEGYTASQPLYQPSHATEQRPQKEPIDQIQATISLNTDQTTASSSLPAASQPQVFQAGTSKPLHSSGINVNAAPFQSMQTVFNMNAPVPPVNEPETLKQQNQYQASYNQSFSSQPHQVEQTELQQEQLQTVVGTYHGSPDQSHQVTGNHQQPPQQNTGFPRSNQPYYNSRGVSRGGSRGARGLMNGYRGPANGFRGGYDGYRPSFSNTPNSGYTQSQFSAPRDYSGYQRDGYQQNFKRGSGQSGPRGAPRGRGGPPRPNRGMPQMNTQQVN",
# SPRL1_HUMAN
"MKTGLFFLCLLGTAAAIPTNARLLSDHSKPTAETVAPDNTAIPSLRAEAEENEKETAVSTEDDSHHKAEKSSVLKSKEESHEQSAEQGKSSSQELGLKDQEDSDGHLSVNLEYAPTEGTLDIKEDMSEPQEKKLSENTDFLAPGVSSFTDSNQQESITKREENQEQPRNYSHHQLNRSSKHSQGLRDQGNQEQDPNISNGEEEEEKEPGEVGTHNDNQERKTELPREHANSKQEEDNTQSDDILEESDQPTQVSKMQEDEFDQGNQEQEDNSNAEMEEENASNVNKHIQETEWQSQEGKTGLEAISNHKETEEKTVSEALLMEPTDDGNTTPRNHGVDDDGDDDGDDGGTDGPRHSASDDYFIPSQAFLEAERAQSIAYHLKIEEQREKVHENENIGTTEPGEHQEAKKAENSSNEEETSSEGNMRVHAVDSCMSFQCKRGHICKADQQGKPHCVCQDPVTCPPTKPLDQVCGTDNQTYASSCHLFATKCRLEGTKKGHQLQLDYFGACKSIPTCTDFEVIQFPLRMRDWLKNILMQLYEANSEHAGYLNEKQRNKVKKIYLDEKRLLAGDHPIDLLLRDFKKNYHMYVYPVHWQFSELDQHPMDRVLTHSELAPLRASLVPMEHCITRFFEECDPNKDKHITLKEWGHCFGIKEEDIDENLLF",
# SEM3A_HUMAN
"MGWLTRIVCLFWGVLLTARANYQNGKNNVPRLKLSYKEMLESNNVITFNGLANSSSYHTFLLDEERSRLYVGAKDHIFSFDLVNIKDFQKIVWPVSYTRRDECKWAGKDILKECANFIKVLKAYNQTHLYACGTGAFHPICTYIEIGHHPEDNIFKLENSHFENGRGKSPYDPKLLTASLLIDGELYSGTAADFMGRDFAIFRTLGHHHPIRTEQHDSRWLNDPKFISAHLISESDNPEDDKVYFFFRENAIDGEHSGKATHARIGQICKNDFGGHRSLVNKWTTFLKARLICSVPGPNGIDTHFDELQDVFLMNFKDPKNPVVYGVFTTSSNIFKGSAVCMYSMSDVRRVFLGPYAHRDGPNYQWVPYQGRVPYPRPGTCPSKTFGGFDSTKDLPDDVITFARSHPAMYNPVFPMNNRPIVIKTDVNYQFTQIVVDRVDAEDGQYDVMFIGTDVGTVLKVVSIPKETWYDLEEVLLEEMTVFREPTAISAMELSTKQQQLYIGSTAGVAQLPLHRCDIYGKACAECCLARDPYCAWDGSACSRYFPTAKRRTRRQDIRNGDPLTHCSDLHHDNHHGHSPEERIIYGVENSSTFLECSPKSQRALVYWQFQRRNEERKEEIRVDDHIIRTDQGLLLRSLQQKDSGNYLCHAVEHGFIQTLLKVTLEVIDTEHLEELLHKDDDGDGSKTKEMSNSMTPSQKVWYRDFMQLINHPNLNTMDEFCEQVWKRDRKQRRQRPGHTPGNSNKWKHLQENKKGRNRRTHEFERAPRSV",
# ITPR3_HUMAN
"MSEMSSFLHIGDIVSLYAEGSVNGFISTLGLVDDRCVVEPAAGDLDNPPKKFRDCLFKVCPMNRYSAQKQYWKAKQTKQDKEKIADVVLLQKLQHAAQMEQKQNDTENKKVHGDVVKYGSVIQLLHMKSNKYLTVNKRLPALLEKNAMRVTLDATGNEGSWLFIQPFWKLRSNGDNVVVGDKVILNPVNAGQPLHASNYELSDNAGCKEVNSVNCNTSWKINLFMQFRDHLEEVLKGGDVVRLFHAEQEKFLTCDEYKGKLQVFLRTTLRQSATSATSSNALWEVEVVHHDPCRGGAGHWNGLYRFKHLATGNYLAAEENPSYKGDASDPKAAGMGAQGRTGRRNAGEKIKYCLVAVPHGNDIASLFELDPTTLQKTDSFVPRNSYVRLRHLCTNTWIQSTNVPIDIEEERPIRLMLGTCPTKEDKEAFAIVSVPVSEIRDLDFANDASSMLASAVEKLNEGFISQNDRRFVIQLLEDLVFFVSDVPNNGQNVLDIMVTKPNRERQKLMREQNILKQVFGILKAPFREKGGEGPLVRLEELSDQKNAPYQHMFRLCYRVLRHSQEDYRKNQEHIAKQFGMMQSQIGYDILAEDTITALLHNNRKLLEKHITKTEVETFVSLVRKNREPRFLDYLSDLCVSNHIAIPVTQELICKCVLDPKNSDILIRTELRPVKEMAQSHEYLSIEYSEEEVWLTWTDKNNEHHEKSVRQLAQEARAGNAHDENVLSYYRYQLKLFARMCLDRQYLAIDEISQQLGVDLIFLCMADEMLPFDLRASFCHLMLHVHVDRDPQELVTPVKFARLWTEIPTAITIKDYDSNLNASRDDKKNKFANTMEFVEDYLNNVVSEAVPFANEEKNKLTFEVVSLAHNLIYFGFYSFSELLRLTRTLLGIIDCVQGPPAMLQAYEDPGGKNVRRSIQGVGHMMSTMVLSRKQSVFSAPSLSAGASAAEPLDRSKFEENEDIVVMETKLKILEILQFILNVRLDYRISYLLSVFKKEFVEVFPMQDSGADGTAPAFDSTTANMNLDRIGEQAEAMFGVGKTSSMLEVDDEGGRMFLRVLIHLTMHDYAPLVSGALQLLFKHFSQRQEAMHTFKQVQLLISAQDVENYKVIKSELDRLRTMVEKSELWVDKKGSGKGEEVEAGAAKDKKERPTDEEGFLHPPGEKSSENYQIVKGILERLNKMCGVGEQMRKKQQRLLKNMDAHKVMLDLLQIPYDKGDAKMMEILRYTHQFLQKFCAGNPGNQALLHKHLHLFLTPGLLEAETMQHIFLNNYQLCSEISEPVLQHFVHLLATHGRHVQYLDFLHTVIKAEGKYVKKCQDMIMTELTNAGDDVVVFYNDKASLAHLLDMMKAARDGVEDHSPLMYHISLVDLLAACAEGKNVYTEIKCTSLLPLEDVVSVVTHEDCITEVKMAYVNFVNHCYVDTEVEMKEIYTSNHIWTLFENFTLDMARVCSKREKRVADPTLEKYVLSVVLDTINAFFSSPFSENSTSLQTHQTIVVQLLQSTTRLLECPWLQQQHKGSVEACIRTLAMVAKGRAILLPMDLDAHISSMLSSGASCAAAAQRNASSYKATTRAFPRVTPTANQWDYKNIIEKLQDIITALEERLKPLVQAELSVLVDVLHWPELLFLEGSEAYQRCESGGFLSKLIQHTKDLMESEEKLCIKVLRTLQQMLLKKTKYGDRGNQLRKMLLQNYLQNRKSTSRGDLPDPIGTGLDPDWSAIAATQCRLDKEGATKLVCDLITSTKNEKIFQESIGLAIHLLDGGNTEIQKSFHNLMMSDKKSERFFKVLHDRMKRAQQETKSTVAVNMNDLGSQPHEDREPVDPTTKGRVASFSIPGSSSRYSLGPSLRRGHEVSERVQSSEMGTSVLIMQPILRFLQLLCENHNRDLQNFLRCQNNKTNYNLVCETLQFLDIMCGSTTGGLGLLGLYINEDNVGLVIQTLETLTEYCQGPCHENQTCIVTHESNGIDIITALILNDISPLCKYRMDLVLQLKDNASKLLLALMESRHDSENAERILISLRPQELVDVIKKAYLQEEERENSEVSPREVGHNIYILALQLSRHNKQLQHLLKPVKRIQEEEAEGISSMLSLNNKQLSQMLKSSAPAQEEEEDPLAYYENHTSQIEIVRQDRSMEQIVFPVPGICQFLTEETKHRLFTTTEQDEQGSKVSDFFDQSSFLHNEMEWQRKLRSMPLIYWFSRRMTLWGSISFNLAVFINIIIAFFYPYMEGASTGVLDSPLISLLFWILICFSIAALFTKRYSIRPLIVALILRSIYYLGIGPTLNILGALNLTNKIVFVVSFVGNRGTFIRGYKAMVMDMEFLYHVGYILTSVLGLFAHELFYSILLFDLIYREETLFNVIKSVTRNGRSILLTALLALILVYLFSIVGFLFLKDDFILEVDRLPNNHSTASPLGMPHGAAAFVDTCSGDKMDCVSGLSVPEVLEEDRELDSTERACDTLLMCIVTVMNHGLRNGGGVGDILRKPSKDESLFPARVVYDLLFFFIVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVSFEEHIKLEHNMWNYLYFIVLVRVKNKTDYTGPESYVAQMIKNKNLDWFPRMRAMSLVSNEGEGEQNEIRILQDKLNSTMKLVSHLTAQLNELKEQMTEQRKRRQRLGFVDVQNCISR",
# ITPR1_HUMAN
"MSDKMSSFLHIGDICSLYAEGSTNGFISTLGLVDDRCVVQPETGDLNNPPKKFRDCLFKLCPMNRYSAQKQFWKAAKPGANSTTDAVLLNKLHHAADLEKKQNETENRKLLGTVIQYGNVIQLLHLKSNKYLTVNKRLPALLEKNAMRVTLDEAGNEGSWFYIQPFYKLRSIGDSVVIGDKVVLNPVNAGQPLHASSHQLVDNPGCNEVNSVNCNTSWKIVLFMKWSDNKDDILKGGDVVRLFHAEQEKFLTCDEHRKKQHVFLRTTGRQSATSATSSKALWEVEVVQHDPCRGGAGYWNSLFRFKHLATGHYLAAEVDPDFEEECLEFQPSVDPDQDASRSRLRNAQEKMVYSLVSVPEGNDISSIFELDPTTLRGGDSLVPRNSYVRLRHLCTNTWVHSTNIPIDKEEEKPVMLKIGTSPVKEDKEAFAIVPVSPAEVRDLDFANDASKVLGSIAGKLEKGTITQNERRSVTKLLEDLVYFVTGGTNSGQDVLEVVFSKPNRERQKLMREQNILKQIFKLLQAPFTDCGDGPMLRLEELGDQRHAPFRHICRLCYRVLRHSQQDYRKNQEYIAKQFGFMQKQIGYDVLAEDTITALLHNNRKLLEKHITAAEIDTFVSLVRKNREPRFLDYLSDLCVSMNKSIPVTQELICKAVLNPTNADILIETKLVLSRFEFEGVSSTGENALEAGEDEEEVWLFWRDSNKEIRSKSVRELAQDAKEGQKEDRDVLSYYRYQLNLFARMCLDRQYLAINEISGQLDVDLILRCMSDENLPYDLRASFCRLMLHMHVDRDPQEQVTPVKYARLWSEIPSEIAIDDYDSSGASKDEIKERFAQTMEFVEEYLRDVVCQRFPFSDKEKNKLTFEVVNLARNLIYFGFYNFSDLLRLTKILLAILDCVHVTTIFPISKMAKGEENKGNNDVEKLKSSNVMRSIHGVGELMTQVVLRGGGFLPMTPMAAAPEGNVKQAEPEKEDIMVMDTKLKIIEILQFILNVRLDYRISCLLCIFKREFDESNSQTSETSSGNSSQEGPSNVPGALDFEHIEEQAEGIFGGSEENTPLDLDDHGGRTFLRVLLHLTMHDYPPLVSGALQLLFRHFSQRQEVLQAFKQVQLLVTSQDVDNYKQIKQDLDQLRSIVEKSELWVYKGQGPDETMDGASGENEHKKTEEGNNKPQKHESTSSYNYRVVKEILIRLSKLCVQESASVRKSRKQQQRLLRNMGAHAVVLELLQIPYEKAEDTKMQEIMRLAHEFLQNFCAGNQQNQALLHKHINLFLNPGILEAVTMQHIFMNNFQLCSEINERVVQHFVHCIETHGRNVQYIKFLQTIVKAEGKFIKKCQDMVMAELVNSGEDVLVFYNDRASFQTLIQMMRSERDRMDENSPLMYHIHLVELLAVCTEGKNVYTEIKCNSLLPLDDIVRVVTHEDCIPEVKIAYINFLNHCYVDTEVEMKEIYTSNHMWKLFENFLVDICRACNNTSDRKHADSILEKYVTEIVMSIVTTFFSSPFSDQSTTLQTRQPVFVQLLQGVFRVYHCNWLMPSQKASVESCIRVLSDVAKSRAIAIPVDLDSQVNNLFLKSHSIVQKTAMNWRLSARNAARRDSVLAASRDYRNIIERLQDIVSALEDRLRPLVQAELSVLVDVLHRPELLFPENTDARRKCESGGFICKLIKHTKQLLEENEEKLCIKVLQTLREMMTKDRGYGEKLISIDELDNAELPPAPDSENATEELEPSPPLRQLEDHKRGEALRQVLVNRYYGNVRPSGRRESLTSFGNGPLSAGGPGKPGGGGGGSGSSSMSRGEMSLAEVQCHLDKEGASNLVIDLIMNASSDRVFHESILLAIALLEGGNTTIQHSFFCRLTEDKKSEKFFKVFYDRMKVAQQEIKATVTVNTSDLGNKKKDDEVDRDAPSRKKAKEPTTQITEEVRDQLLEASAATRKAFTTFRREADPDDHYQPGEGTQATADKAKDDLEMSAVITIMQPILRFLQLLCENHNRDLQNFLRCQNNKTNYNLVCETLQFLDCICGSTTGGLGLLGLYINEKNVALINQTLESLTEYCQGPCHENQNCIATHESNGIDIITALILNDINPLGKKRMDLVLELKNNASKLLLAIMESRHDSENAERILYNMRPKELVEVIKKAYMQGEVEFEDGENGEDGAASPRNVGHNIYILAHQLARHNKELQSMLKPGGQVDGDEALEFYAKHTAQIEIVRLDRTMEQIVFPVPSICEFLTKESKLRIYYTTERDEQGSKINDFFLRSEDLFNEMNWQKKLRAQPVLYWCARNMSFWSSISFNLAVLMNLLVAFFYPFKGVRGGTLEPHWSGLLWTAMLISLAIVIALPKPHGIRALIASTILRLIFSVGLQPTLFLLGAFNVCNKIIFLMSFVGNCGTFTRGYRAMVLDVEFLYHLLYLVICAMGLFVHEFFYSLLLFDLVYREETLLNVIKSVTRNGRSIILTAVLALILVYLFSIVGYLFFKDDFILEVDRLPNETAVPETGESLASEFLFSDVCRVESGENCSSPAPREELVPAEETEQDKEHTCETLLMCIVTVLSHGLRSGGGVGDVLRKPSKEEPLFAARVIYDLLFFFMVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVTFEEHIKEEHNMWHYLCFIVLVKVKDSTEYTGPESYVAEMIKERNLDWFPRMRAMSLVSSDSEGEQNELRNLQEKLESTMKLVTNLSGQLSELKDQMTEQRKQKQRIGLLGHPPHMNVNPQQPA",
# KCJ11_HUMAN
"MLSRKGIIPEEYVLTRLAEDPAKPRYRARQRRARFVSKKGNCNVAHKNIREQGRFLQDVFTTLVDLKWPHTLLIFTMSFLCSWLLFAMAWWLIAFAHGDLAPSEGTAEPCVTSIHSFSSAFLFSIEVQVTIGFGGRMVTEECPLAILILIVQNIVGLMINAIMLGCIFMKTAQAHRRAETLIFSKHAVIALRHGRLCFMLRVGDLRKSMIISATIHMQVVRKTTSPEGEVVPLHQVDIPMENGVGGNSIFLVAPLIIYHVIDANSPLYDLAPSDLHHHQDLEIIVILEGVVETTGITTQARTSYLADEILWGQRFVPIVAEEDGRYSVDYSKFGNTVKVPTPLCTARQLDEDHSLLEALTLASARGPLRKRSVPMAKAKPKFSISPDSLS",
# DIP2A_HUMAN
"MADRGCPLEAAPLPAEVRESLAELELELSEGDITQKGYEKKRAKLLARYIPLIQGIDPSLQAENRIPGPSQTTAAAPKQQKSRPTASRDERFRSDVHTEAVQAALAKYKERKMPMPSKRRSVLVHSSVETYTPPDTSSASEDEGSLRRPGRLTSTPLQSHSSVEPWLDRVIQGSSTSSSASSTSSHPGGRPTTAPSAAATPGAAATTALAGLEAHTHIDLHSAPPDVTTGLVEHSYFERPQVASVRSVPRGCSGSMLETADGVPVNSRVSSKIQQLLNTLKRPKRPPLKEFFVDDFEELLEVQQPDPNQPKPEGSETSVLRGEPLTAGVPRPPSLLATLQRWGTTQPKSPCLTALDTTGKAVYTLTYGKLWSRSLKLAYTLLNKLTSKNEPLLKPGDRVALVFPNSDPVMFMVAFYGCLLAELVPVPIEVPLTRKDAGSQQVGFLLGSCGVFLALTTDACQKGLPKAQTGEVAAFKGWPPLSWLVIDGKHLAKPPKDWHPLAQDTGTGTAYIEYKTSKEGSTVGVTVSHASLLAQCRALTQACGYSEAETLTNVLDFKRDAGLWHGVLTSVMNRMHVVSVPYALMKANPLSWIQKVCFYKARAALVKSRDMHWSLLAQRGQRDVSLSSLRMLIVADGANPWSISSCDAFLNVFQSRGLRPEVICPCASSPEALTVAIRRPPDLGGPPPRKAVLSMNGLSYGVIRVDTEEKLSVLTVQDVGQVMPGANVCVVKLEGTPYLCKTDEVGEICVSSSATGTAYYGLLGITKNVFEAVPVTTGGAPIFDRPFTRTGLLGFIGPDNLVFIVGKLDGLMVTGVRRHNADDVVATALAVEPMKFVYRGRIAVFSVTVLHDDRIVLVAEQRPDASEEDSFQWMSRVLQAIDSIHQVGVYCLALVPANTLPKAPLGGIHISETKQRFLEGTLHPCNVLMCPHTCVTNLPKPRQKQPEVGPASMIVGNLVAGKRIAQASGRELAHLEDSDQARKFLFLADVLQWRAHTTPDHPLFLLLNAKGTVTSTATCVQLHKRAERVAAALMEKGRLSVGDHVALVYPPGVDLIAAFYGCLYCGCVPVTVRPPHPQNLGTTLPTVKMIVEVSKSACVLTTQAVTRLLRSKEAAAAVDIRTWPTILDTDDIPKKKIASVFRPPSPDVLAYLDFSVSTTGILAGVKMSHAATSALCRSIKLQCELYPSRQIAICLDPYCGLGFALWCLCSVYSGHQSVLVPPLELESNVSLWLSAVSQYKARVTFCSYSVMEMCTKGLGAQTGVLRMKGVNLSCVRTCMVVAEERPRIALTQSFSKLFKDLGLPARAVSTTFGCRVNVAICLQGTAGPDPTTVYVDMRALRHDRVRLVERGSPHSLPLMESGKILPGVKVIIAHTETKGPLGDSHLGEIWVSSPHNATGYYTVYGEEALHADHFSARLSFGDTQTIWARTGYLGFLRRTELTDASGGRHDALYVVGSLDETLELRGMRYHPIDIETSVIRAHRSIAECAVFTWTNLLVVVVELDGLEQDALDLVALVTNVVLEEHYLVVGVVVIVDPGVIPINSRGEKQRMHLRDGFLADQLDPIYVAYNM",
# KCNB1_HUMAN
"MPAGMTKHGSRSTSSLPPEPMEIVRSKACSRRVRLNVGGLAHEVLWRTLDRLPRTRLGKLRDCNTHDSLLEVCDDYSLDDNEYFFDRHPGAFTSILNFYRTGRLHMMEEMCALSFSQELDYWGIDEIYLESCCQARYHQKKEQMNEELKREAETLREREGEEFDNTCCAEKRKKLWDLLEKPNSSVAAKILAIISIMFIVLSTIALSLNTLPELQSLDEFGQSTDNPQLAHVEAVCIAWFTMEYLLRFLSSPKKWKFFKGPLNAIDLLAILPYYVTIFLTESNKSVLQFQNVRRVVQIFRIMRILRILKLARHSTGLQSLGFTLRRSYNELGLLILFLAMGIMIFSSLVFFAEKDEDDTKFKSIPASFWWATITMTTVGYGDIYPKTLLGKIVGGLCCIAGVLVIALPIPIIVNNFSEFYKEQKRQEKAIKRREALERAKRNGSIVSMNMKDAFARSIEMMDIVVEKNGENMGKKDKVQDNHLSPNKWKWTKRTLSETSSSKSFETKEQGSPEKARSSSSPQHLNVQQLEDMYNKMAKTQSQPILNTKESAAQSKPKEELEMESIPSPVAPLPTRTEGVIDMRSMSSIDSFISCATDFPEATRFSHSPLTSLPSKTGGSTAPEVGWRGALGASGGRFVEANPSPDASQHSSFFIESPKSSMKTNNPLKLRALKVNFMEGDPSPLLPVLGMYHDPLRNRGSAAAAVAGLECATLLDKAVLSPESSIYTTASAKTPPRSPEKHTAIAFNFEAGVHQYIDADTDDEGQLLYSVDSSPPKSLPGSTSPKFSTGTRSEKNHFESSPLPTSPKFLRQNCIYSTEALTGKGPSGQEKCKLENHISPDVRVLPGGGAHGSTRDQSI",
# GRM7_HUMAN
"MVQLRKLLRVLTLMKFPCCVLEVLLCALAAAARGQEMYAPHSIRIEGDVTLGGLFPVHAKGPSGVPCGDIKRENGIHRLEAMLYALDQINSDPNLLPNVTLGARILDTCSRDTYALEQSLTFVQALIQKDTSDVRCTNGEPPVFVKPEKVVGVIGASGSSVSIMVANILRLFQIPQISYASTAPELSDDRRYDFFSRVVPPDSFQAQAMVDIVKALGWNYVSTLASEGSYGEKGVESFTQISKEAGGLCIAQSVRIPQERKDRTIDFDRIIKQLLDTPNSRAVVIFANDEDIKQILAAAKRADQVGHFLWVGSDSWGSKINPLHQHEDIAEGAITIQPKRATVEGFDAYFTSRTLENNRRNVWFAEYWEENFNCKLTISGSKKEDTDRKCTGQERIGKDSNYEQEGKVQFVIDAVYAMAHALHHMNKDLCADYRGVCPEMEQAGGKKLLKYIRNVNFNGSAGTPVMFNKNGDAPGRYDIFQYQTTNTSNPGYRLIGQWTDELQLNIEDMQWGKGVREIPASVCTLPCKPGQRKKTQKGTPCCWTCEPCDGYQYQFDEMTCQHCPYDQRPNENRTGCQDIPIIKLEWHSPWAVIPVFLAMLGIIATIFVMATFIRYNDTPIVRASGRELSYVLLTGIFLCYIITFLMIAKPDVAVCSFRRVFLGLGMCISYAALLTKTNRIYRIFEQGKKSVTAPRLISPTSQLAITSSLISVQLLGVFIWFGVDPPNIIIDYDEHKTMNPEQARGVLKCDITDLQIICSLGYSILLMVTCTVYAIKTRGVPENFNEAKPIGFTMYTTCIVWLAFIPIFFGTAQSAEKLYIQTTTLTISMNLSASVALGMLYMPKVYIIIFHPELNVQKRKRSFKAVVTAATMSSRLSHKPSDRPNGEAKTELCENVDPNSPAAKKKYVSYNNLVI",
# GRM3_HUMAN
"MKMLTRLQVLTLALFSKGFLLSLGDHNFLRREIKIEGDLVLGGLFPINEKGTGTEECGRINEDRGIQRLEAMLFAIDEINKDDYLLPGVKLGVHILDTCSRDTYALEQSLEFVRASLTKVDEAEYMCPDGSYAIQENIPLLIAGVIGGSYSSVSIQVANLLRLFQIPQISYASTSAKLSDKSRYDYFARTVPPDFYQAKAMAEILRFFNWTYVSTVASEGDYGETGIEAFEQEARLRNICIATAEKVGRSNIRKSYDSVIRELLQKPNARVVVLFMRSDDSRELIAAASRANASFTWVASDGWGAQESIIKGSEHVAYGAITLELASQPVRQFDRYFQSLNPYNNHRNPWFRDFWEQKFQCSLQNKRNHRRVCDKHLAIDSSNYEQESKIMFVVNAVYAMAHALHKMQRTLCPNTTKLCDAMKILDGKKLYKDYLLKINFTAPFNPNKDADSIVKFDTFGDGMGRYNVFNFQNVGGKYSYLKVGHWAETLSLDVNSIHWSRNSVPTSQCSDPCAPNEMKNMQPGDVCCWICIPCEPYEYLADEFTCMDCGSGQWPTADLTGCYDLPEDYIRWEDAWAIGPVTIACLGFMCTCMVVTVFIKHNNTPLVKASGRELCYILLFGVGLSYCMTFFFIAKPSPVICALRRLGLGSSFAICYSALLTKTNCIARIFDGVKNGAQRPKFISPSSQVFICLGLILVQIVMVSVWLILEAPGTRRYTLAEKRETVILKCNVKDSSMLISLTYDVILVILCTVYAFKTRKCPENFNEAKFIGFTMYTTCIIWLAFLPIFYVTSSDYRVQTTTMCISVSLSGFVVLGCLFAPKVHIILFQPQKNVVTHRLHLNRFSVSGTGTTYSQSSASTYVPTVCNGREVLDSTTSSL",
# GRM4_HUMAN
"MPGKRGLGWWWARLPLCLLLSLYGPWMPSSLGKPKGHPHMNSIRIDGDITLGGLFPVHGRGSEGKPCGELKKEKGIHRLEAMLFALDRINNDPDLLPNITLGARILDTCSRDTHALEQSLTFVQALIEKDGTEVRCGSGGPPIITKPERVVGVIGASGSSVSIMVANILRLFKIPQISYASTAPDLSDNSRYDFFSRVVPSDTYQAQAMVDIVRALKWNYVSTVASEGSYGESGVEAFIQKSREDGGVCIAQSVKIPREPKAGEFDKIIRRLLETSNARAVIIFANEDDIRRVLEAARRANQTGHFFWMGSDSWGSKIAPVLHLEEVAEGAVTILPKRMSVRGFDRYFSSRTLDNNRRNIWFAEFWEDNFHCKLSRHALKKGSHVKKCTNRERIGQDSAYEQEGKVQFVIDAVYAMGHALHAMHRDLCPGRVGLCPRMDPVDGTQLLKYIRNVNFSGIAGNPVTFNENGDAPGRYDIYQYQLRNDSAEYKVIGSWTDHLHLRIERMHWPGSGQQLPRSICSLPCQPGERKKTVKGMPCCWHCEPCTGYQYQVDRYTCKTCPYDMRPTENRTGCRPIPIIKLEWGSPWAVLPLFLAVVGIAATLFVVITFVRYNDTPIVKASGRELSYVLLAGIFLCYATTFLMIAEPDLGTCSLRRIFLGLGMSISYAALLTKTNRIYRIFEQGKRSVSAPRFISPASQLAITFSLISLQLLGICVWFVVDPSHSVVDFQDQRTLDPRFARGVLKCDISDLSLICLLGYSMLLMVTCTVYAIKTRGVPETFNEAKPIGFTMYTTCIVWLAFIPIFFGTSQSADKLYIQTTTLTVSVSLSASVSLGMLYMPKVYIILFHPEQNVPKRKRSLKAVVTAATMSNKFTQKGNFRPNGEAKSELCENLEAPALATKQTYVTYTNHAI",
# CHD4_HUMAN
"MASGLGSPSPCSAGSEEEDMDALLNNSLPPPHPENEEDPEEDLSETETPKLKKKKKPKKPRDPKIPKSKRQKKERMLLCRQLGDSSGEGPEFVEEEEEVALRSDSEGSDYTPGKKKKKKLGPKKEKKSKSKRKEEEEEEDDDDDSKEPKSSAQLLEDWGMEDIDHVFSEEDYRTLTNYKAFSQFVRPLIAAKNPKIAVSKMMMVLGAKWREFSTNNPFKGSSGASVAAAAAAAVAVVESMVTATEVAPPPPPVEVPIRKAKTKEGKGPNARRKPKGSPRVPDAKKPKPKKVAPLKIKLGGFGSKRKRSSSEDDDLDVESDFDDASINSYSVSDGSTSRSSRSRKKLRTTKKKKKGEEEVTAVDGYETDHQDYCEVCQQGGEIILCDTCPRAYHMVCLDPDMEKAPEGKWSCPHCEKEGIQWEAKEDNSEGEEILEEVGGDLEEEDDHHMEFCRVCKDGGELLCCDTCPSSYHIHCLNPPLPEIPNGEWLCPRCTCPALKGKVQKILIWKWGQPPSPTPVPRPPDADPNTPSPKPLEGRPERQFFVKWQGMSYWHCSWVSELQLELHCQVMFRNYQRKNDMDEPPSGDFGGDEEKSRKRKNKDPKFAEMEERFYRYGIKPEWMMIHRILNHSVDKKGHVHYLIKWRDLPYDQASWESEDVEIQDYDLFKQSYWNHRELMRGEEGRPGKKLKKVKLRKLERPPETPTVDPTVKYERQPEYLDATGGTLHPYQMEGLNWLRFSWAQGTDTILADEMGLGKTVQTAVFLYSLYKEGHSKGPFLVSAPLSTIINWEREFEMWAPDMYVVTYVGDKDSRAIIRENEFSFEDNAIRGGKKASRMKKEASVKFHVLLTSYELITIDMAILGSIDWACLIVDEAHRLKNNQSKFFRVLNGYSLQHKLLLTGTPLQNNLEELFHLLNFLTPERFHNLEGFLEEFADIAKEDQIKKLHDMLGPHMLRRLKADVFKNMPSKTELIVRVELSPMQKKYYKYILTRNFEALNARGGGNQVSLLNVVMDLKKCCNHPYLFPVAAMEAPKMPNGMYDGSALIRASGKLLLLQKMLKNLKEGGHRVLIFSQMTKMLDLLEDFLEHEGYKYERIDGGITGNMRQEAIDRFNAPGAQQFCFLLSTRAGGLGINLATADTVIIYDSDWNPHNDIQAFSRAHRIGQNKKVMIYRFVTRASVEERITQVAKKKMMLTHLVVRPGLGSKTGSMSKQELDDILKFGTEELFKDEATDGGGDNKEGEDSSVIHYDDKAIERLLDRNQDETEDTELQGMNEYLSSFKVAQYVVREEEMGEEEEVEREIIKQEESVDPDYWEKLLRHHYEQQQEDLARNLGKGKRIRKQVNYNDGSQEDRDWQDDQSDNQSDYSVASEEGDEDFDERSEAPRRPSRKGLRNDKDKPLPPLLARVGGNIEVLGFNARQRKAFLNAIMRYGMPPQDAFTTQWLVRDLRGKSEKEFKAYVSLFMRHLCEPGADGAETFADGVPREGLSRQHVLTRIGVMSLIRKKVQEFEHVNGRWSMPELAEVEENKKMSQPGSPSPKTPTPSTPGDTQPNTPAPVPPAEDGIKIEENSLKEEESIEGEKEVKSTAPETAIECTQAPAPASEDEKVVVEPPEGEEKVEKAEVKERTEEPMETEPKGAADVEKVEEKSAIDLTPIVVEDKEEKKEEEEKKEVMLQNGETPKDLNDEKQKKNIKQRFMFNIADGGFTELHSLWQNEERAATVTKKTYEIWHRRHDYWLLAGIINHGYARWQDIQNDPRYAILNEPFKGEMNRGNFLEIKNKFLARRFKLLEQALVIEEQLRRAAYLNMSEDPSHPSMALNTRFAEVECLAESHQHLSKESMAGNKPANAVLHKVLKQLEELLSDMKADVTRLPATIARIPPVAVRLQMSERNILSRLANRAPEPTPQQVAQQQ",
# LASP1_HUMAN
"MNPNCARCGKIVYPTEKVNCLDKFWHKACFHCETCKMTLNMKNYKGYEKKPYCNAHYPKQSFTMVADTPENLRLKQQSELQSQVRYKEEFEKNKGKGFSVVADTPELQRIKKTQDQISNIKYHEEFEKSRMGPSGGEGMEPERRDSQDGSSYRRPLEQQQPHHIPTSAPVYQQPQQQPVAQSYGGYKEPAAPVSIQRSAPGGGGKRYRAVYDYSAADEDEVSFQDGDTIVNVQQIDDGWMYGTVERTGDTGMLPANYVEAI",
# NMDE3_HUMAN
"MGGALGPALLLTSLFGAWAGLGPGQGEQGMTVAVVFSSSGPPQAQFRARLTPQSFLDLPLEIQPLTVGVNTTNPSSLLTQICGLLGAAHVHGIVFEDNVDTEAVAQILDFISSQTHVPILSISGGSAVVLTPKEPGSAFLQLGVSLEQQLQVLFKVLEEYDWSAFAVITSLHPGHALFLEGVRAVADASHVSWRLLDVVTLELGPGGPRARTQRLLRQLDAPVFVAYCSREEAEVLFAEAAQAGLVGPGHVWLVPNLALGSTDAPPATFPVGLISVVTESWRLSLRQKVRDGVAILALGAHSYWRQHGTLPAPAGDCRVHPGPVSPAREAFYRHLLNVTWEGRDFSFSPGGYLVQPTMVVIALNRHRLWEMVGRWEHGVLYMKYPVWPRYSASLQPVVDSRHLTVATLEERPFVIVESPDPGTGGCVPNTVPCRRQSNHTFSSGDVAPYTKLCCKGFCIDILKKLARVVKFSYDLYLVTNGKHGKRVRGVWNGMIGEVYYKRADMAIGSLTINEERSEIVDFSVPFVETGISVMVARSNGTVSPSAFLEPYSPAVWVMMFVMCLTVVAITVFMFEYFSPVSYNQNLTRGKKSGGPAFTIGKSVWLLWALVFNNSVPIENPRGTTSKIMVLVWAFFAVIFLASYTANLAAFMIQEQYIDTVSGLSDKKFQRPQDQYPPFRFGTVPNGSTERNIRSNYRDMHTHMVKFNQRSVEDALTSLKMGKLDAFIYDAAVLNYMAGKDEGCKLVTIGSGKVFATTGYGIAMQKDSHWKRAIDLALLQFLGDGETQKLETVWLSGICQNEKNEVMSSKLDIDNMAGVFYMLLVAMGLALLVFAWEHLVYWKLRHSVPNSSQLDFLLAFSRGIYSCFSGVQSLASPPRQASPDLTASSAQASVLKMLQAARDMVTTAGVSSSLDRATRTIENWGGGRRAPPPSPCPTPRSGPSPCLPTPDPPPEPSPTGWGPPDGGRAALVRRAPQPPGRPPTPGPPLSDVSRVSRRPAWEARWPVRTGHCGRHLSASERPLSPARCHYSSFPRADRSGRPFLPLFPELEDLPLLGPEQLARREALLHAAWARGSRPRHASLPSSVAEAFARPSSLPAGCTGPACARPDGHSACRRLAQAQSMCLPIYREACQEGEQAGAPAWQHRQHVCLHAHAHLPFCWGAVCPHLPPCASHGSWLSGAWGPLGHRGRTLGLGTGYRDSGGLDEISRVARGTQGFPGPCTWRRISSLESEV",
# OPCM_HUMAN
"MGVCGYLFLPWKCLVVVSLRLLFLVPTGVPVRSGDATFPKAMDNVTVRQGESATLRCTIDDRVTRVAWLNRSTILYAGNDKWSIDPRVIILVNTPTQYSIMIQNVDVYDEGPYTCSVQTDNHPKTSRVHLIVQVPPQIMNISSDITVNEGSSVTLLCLAIGRPEPTVTWRHLSVKEGQGFVSEDEYLEISDIKRDQSGEYECSALNDVAAPDVRKVKITVNYPPYISKAKNTGVSVGQKGILSCEASAVPMAEFQWFKEETRLATGLDGMRIENKGRMSTLTFFNVSEKDYGNYTCVATNKLGNTNASITLYGPGAVIDGVNSASRALACLWLSGTLLAHFFIKF",
# FRPD4_HUMAN
"MDVFSFVKIAKLSSHRTKSSGWPPPSGTWGLSQVPPYGWEMTANRDGRDYFINHMTQAIPFDDPRLESCQIIPPAPRKVEMRRDPVLGFGFVAGSEKPVVVRSVTPGGPSEGKLIPGDQIVMINDEPVSAAPRERVIDLVRSCKESILLTVIQPYPSPKSAFISAAKKARLKSNPVKVRFSEEVIINGQVSETVKDNSLLFMPNVLKVYLENGQTKSFRFDCSTSIKDVILTLQEKLSIKGIEHFSLMLEQRTEGAGTKLLLLHEQETLTQVTQRPSSHKMRCLFRISFVPKDPIDLLRRDPVAFEYLYVQSCNDVVQERFGPELKYDIALRLAALQMYIATVTTKQTQKISLKYIEKEWGLETFLPSAVLQSMKEKNIKKALSHLVKANQNLVPPGKKLSALQAKVHYLKFLSDLRLYGGRVFKATLVQAEKRSEVTLLVGPRYGISHVINTKTNLVALLADFSHVNRIEMFSEEESLVRVELHVLDVKPITLLMESSDAMNLACLTAGYYRLLVDSRRSIFNMANKKNTATQETGPENKGKHNLLGPDWNCIPQMTTFIGEGEQEAQITYIDSKQKTVEITDSTMCPKEHRHLYIDNAYSSDGLNQQLSQPGEAPCEADYRSLAQRSLLTLSGPETLKKAQESPRGAKVSFIFGDFALDDGISPPTLGYETLLDEGPEMLEKQRNLYIGSANDMKGLDLTPEAEGIQFVENSVYANIGDVKSFQAAEGIEEPLLHDICYAENTDDAEDEDEVSCEEDLVVGEMNQPAILNLSGSSDDIIDLTSLPPPEGDDNEDDFLLRSLNMAIAAPPPGFRDSSDEEDSQSQAASFPEDKEKGSSLQNDEIPVSLIDAVPTSAEGKCEKGLDNAVVSTLGALEALSVSEEQQTSDNSGVAILRAYSPESSSDSGNETNSSEMTESSELATAQKQSENLSRMFLATHEGYHPLAEEQTEFPASKTPAGGLPPKSSHALAARPATDLPPKVVPSKQLLHSDHMEMEPETMETKSVTDYFSKLHMGSVAYSCTSKRKSKLADGEGKAPPNGNTTGKKQQGTKTAEMEEEASGKFGTVSSRDSQHLSTFNLERTAFRKDSQRWYVATEGGMAEKSGLEAATGKTFPRASGLGAREAEGKEEGAPDGETSDGSGLGQGDRFLTDVTCASSAKDLDNPEDADSSTCDHPSKLPEADESVARLCDYHLAKRMSSLQSEGHFSLQSSQGSSVDAGCGTGSSGSACATPVESPLCPSLGKHLIPDASGKGVNYIPSEERAPGLPNHGATFKELHPQTEGMCPRMTVPALHTAINTEPLFGTLRDGCHRLPKIKETTV",
# SEPT2_HUMAN
"MSKQQPTQFINPETPGYVGFANLPNQVHRKSVKKGFEFTLMVVGESGLGKSTLINSLFLTDLYPERVIPGAAEKIERTVQIEASTVEIEERGVKLRLTVVDTPGYGDAINCRDCFKTIISYIDEQFERYLHDESGLNRRHIIDNRVHCCFYFISPFGHGLKPLDVAFMKAIHNKVNIVPVIAKADTLTLKERERLKKRILDEIEEHNIKIYHLPDAESDEDEDFKEQTRLLKASIPFSVVGSNQLIEAKGKKVRGRLYPWGVVEVENPEHNDFLKLRTMLITHMQDLQEVTQDLHYENFRSERLKRGGRKVENEDMNKDQILLEKEAELRRMQEMIARMQAQMQMQMQGGDGDGGALGHHV",
# EEA1_HUMAN
"MLRRILQRTPGRVGSQGSDLDSSATPINTVDVNNESSSEGFICPQCMKSLGSADELFKHYEAVHDAGNDSGHGGESNLALKRDDVTLLRQEVQDLQASLKEEKWYSEELKKELEKYQGLQQQEAKPDGLVTDSSAELQSLEQQLEEAQTENFNIKQMKDLFEQKAAQLATEIADIKSKYDEERSLREAAEQKVTRLTEELNKEATVIQDLKTELLQRPGIEDVAVLKKELVQVQTLMDNMTLERERESEKLKDECKKLQSQYASSEATISQLRSELAKGPQEVAVYVQELQKLKSSVNELTQKNQTLTENLLKKEQDYTKLEEKHNEESVSKKNIQATLHQKDLDCQQLQSRLSASETSLHRIHVELSEKGEATQKLKEELSEVETKYQHLKAEFKQLQQQREEKEQHGLQLQSEINQLHSKLLETERQLGEAHGRLKEQRQLSSEKLMDKEQQVADLQLKLSRLEEQLKEKVTNSTELQHQLDKTKQQHQEQQALQQSTTAKLREAQNDLEQVLRQIGDKDQKIQNLEALLQKSKENISLLEKEREDLYAKIQAGEGETAVLNQLQEKNHTLQEQVTQLTEKLKNQSESHKQAQENLHDQVQEQKAHLRAAQDRVLSLETSVNELNSQLNESKEKVSQLDIQIKAKTELLLSAEAAKTAQRADLQNHLDTAQNALQDKQQELNKITTQLDQVTAKLQDKQEHCSQLESHLKEYKEKYLSLEQKTEELEGQIKKLEADSLEVKASKEQALQDLQQQRQLNTDLELRATELSKQLEMEKEIVSSTRLDLQKKSEALESIKQKLTKQEEEKKILKQDFETLSQETKIQHEELNNRIQTTVTELQKVKMEKEALMTELSTVKDKLSKVSDSLKNSKSEFEKENQKGKAAILDLEKTCKELKHQLQVQMENTLKEQKELKKSLEKEKEASHQLKLELNSMQEQLIQAQNTLKQNEKEEQQLQGNINELKQSSEQKKKQIEALQGELKIAVLQKTELENKLQQQLTQAAQELAAEKEKISVLQNNYEKSQETFKQLQSDFYGRESELLATRQDLKSVEEKLSLAQEDLISNRNQIGNQNKLIQELKTAKATLEQDSAKKEQQLQERCKALQDIQKEKSLKEKELVNEKSKLAEIEEIKCRQEKEITKLNEELKSHKLESIKEITNLKDAKQLLIQQKLELQGKADSLKAAVEQEKRNQQILKDQVKKEEEELKKEFIEKEAKLHSEIKEKEVGMKKHEENEAKLTMQITALNENLGTVKKEWQSSQRRVSELEKQTDDLRGEIAVLEATVQNNQDERRALLERCLKGEGEIEKLQTKVLELQRKLDNTTAAVQELGRENQSLQIKHTQALNRKWAEDNEVQNCMACGKGFSVTVRRHHCRQCGNIFCAECSAKNALTPSSKKPVRVCDACFNDLQG",
# CD5R1_HUMAN
"MGTVLSLSPSYRKATLFEDGAATVGHYTAVQNSKNAKDKNLKRHSIISVLPWKRIVAVSAKKKNSKKVQPNSSYQNNITHLNNENLKKSLSCANLSTFAQPPPAQPPAPPASQLSGSQTGGSSSVKKAPHPAVTSAGTPKRVIVQASTSELLRCLGEFLCRRCYRLKHLSPTDPVLWLRSVDRSLLLQGWQDQGFITPANVVFLYMLCRDVISSEVGSDHELQAVLLTCLYLSYSYMGNEISYPLKPFLVESCKEAFWDRCLSVINLMSSKMLQINADPHYFTQVFSDLKNESGQEDKKRLLLGLDR",
# PLCB4_HUMAN
"MAKPYEFNWQKEVPSFLQEGAVFDRYEEESFVFEPNCLFKVDEFGFFLTWRSEGKEGQVLECSLINSIRSGAIPKDPKILAALEAVGKSENDLEGRIVCVCSGTDLVNISFTYMVAENPEVTKQWVEGLRSIIHNFRANNVSPMTCLKKHWMKLAFMTNTNGKIPVRSITRTFASGKTEKVIFQALKELGLPSGKNDEIEPTAFSYEKFYELTQKICPRTDIEDLFKKINGDKTDYLTVDQLVSFLNEHQRDPRLNEILFPFYDAKRAMQIIEMYEPDEDLKKKGLISSDGFCRYLMSDENAPVFLDRLELYQEMDHPLAHYFISSSHNTYLTGRQFGGKSSVEMYRQVLLAGCRCVELDCWDGKGEDQEPIITHGKAMCTDILFKDVIQAIKETAFVTSEYPVILSFENHCSKYQQYKMSKYCEDLFGDLLLKQALESHPLEPGRALPSPNDLKRKILIKNKRLKPEVEKKQLEALRSMMEAGESASPANILEDDNEEEIESADQEEEAHPEFKFGNELSADDLGHKEAVANSVKKGLVTVEDEQAWMASYKYVGATTNIHPYLSTMINYAQPVKFQGFHVAEERNIHYNMSSFNESVGLGYLKTHAIEFVNYNKRQMSRIYPKGGRVDSSNYMPQIFWNAGCQMVSLNYQTPDLAMQLNQGKFEYNGSCGYLLKPDFMRRPDRTFDPFSETPVDGVIAATCSVQVISGQFLSDKKIGTYVEVDMYGLPTDTIRKEFRTRMVMNNGLNPVYNEESFVFRKVILPDLAVLRIAVYDDNNKLIGQRILPLDGLQAGYRHISLRNEGNKPLSLPTIFCNIVLKTYVPDGFGDIVDALSDPKKFLSITEKRADQMRAMGIETSDIADVPSDTSKNDKKGKANTAKANVTPQSSSELRPTTTAALASGVEAKKGIELIPQVRIEDLKQMKAYLKHLKKQQKELNSLKKKHAKEHSTMQKLHCTQVDKIVAQYDKEKSTHEKILEKAMKKKGGSNCLEMKKETEIKIQTLTSDHKSKVKEIVAQHTKEWSEMINTHSAEEQEIRDLHLSQQCELLKKLLINAHEQQTQQLKLSHDRESKEMRAHQAKISMENSKAISQDKSIKNKAERERRVRELNSSNTKKFLEERKRLAMKQSKEMDQLKKVQLEHLEFLEKQNEQAKEMQQMVKLEAEMDRRPATVV",
# STK38_HUMAN
"MAMTGSTPCSSMSNHTKERVTMTKVTLENFYSNLIAQHEEREMRQKKLEKVMEEEGLKDEEKRLRRSAHARKETEFLRLKRTRLGLEDFESLKVIGRGAFGEVRLVQKKDTGHVYAMKILRKADMLEKEQVGHIRAERDILVEADSLWVVKMFYSFQDKLNLYLIMEFLPGGDMMTLLMKKDTLTEEETQFYIAETVLAIDSIHQLGFIHRDIKPDNLLLDSKGHVKLSDFGLCTGLKKAHRTEFYRNLNHSLPSDFTFQNMNSKRKAETWKRNRRQLAFSTVGTPDYIAPEVFMQTGYNKLCDWWSLGVIMYEMLIGYPPFCSETPQETYKKVMNWKETLTFPPEVPISEKAKDLILRFCCEWEHRIGAPGVEEIKSNSFFEGVDWEHIRERPAAISIEIKSIDDTSNFDEFPESDILKPTVATSNHPETDYKNKDWVFINYTYKRFEGLTARGAIPSYMKAAK",
# NECT1_HUMAN
"MARMGLAGAAGRWWGLALGLTAFFLPGVHSQVVQVNDSMYGFIGTDVVLHCSFANPLPSVKITQVTWQKSTNGSKQNVAIYNPSMGVSVLAPYRERVEFLRPSFTDGTIRLSRLELEDEGVYICEFATFPTGNRESQLNLTVMAKPTNWIEGTQAVLRAKKGQDDKVLVATCTSANGKPPSVVSWETRLKGEAEYQEIRNPNGTVTVISRYRLVPSREAHQQSLACIVNYHMDRFKESLTLNVQYEPEVTIEGFDGNWYLQRMDVKLTCKADANPPATEYHWTTLNGSLPKGVEAQNRTLFFKGPINYSLAGTYICEATNPIGTRSGQVEVNITEFPYTPSPPEHGRRAGPVPTAIIGGVAGSILLVLIVVGGIVVALRRRRHTFKGDYSTKKHVYGNGYSKAGIPQHHPPMAQNLQYPDDSDDEKKAGPLGGSSYEEEEEEEEGGGGGERKVGGPHPKYDEDAKRPYFTVDEAEARQDGYGDRTLGYQYDPEQLDLAENMVSQNDGSFISKKEWYV",
# RABE1_HUMAN
"MAQPGPASQPDVSLQQRVAELEKINAEFLRAQQQLEQEFNQKRAKFKELYLAKEEDLKRQNAVLQAAQDDLGHLRTQLWEAQAEMENIKAIATVSENTKQEAIDEVKRQWREEVASLQAVMKETVRDYEHQFHLRLEQERTQWAQYRESAEREIADLRRRLSEGQEEENLENEMKKAQEDAEKLRSVVMPMEKEIAALKDKLTEAEDKIKELEASKVKELNHYLEAEKSCRTDLEMYVAVLNTQKSVLQEDAEKLRKELHEVCHLLEQERQQHNQLKHTWQKANDQFLESQRLLMRDMQRMEIVLTSEQLRQVEELKKKDQEDDEQQRLNKRKDHKKADVEEEIKIPVVCALTQEESSAQLSNEEEHLDSTRGSVHSLDAGLLLPSGDPFSKSDNDMFKDGLRRAQSTDSLGTSGSLQSKALGYNYKAKSAGNLDESDFGPLVGADSVSENFDTASLGSLQMPSGFMLTKDQERAIKAMTPEQEETASLLSSVTQGMESAYVSPSGYRLVSETEWNLLQKEVHNAGNKLGRRCDMCSNYEKQLQGIQIQEAETRDQVKKLQLMLRQANDQLEKTMKDKQELEDFIKQSSEDSSHQISALVLRAQASEILLEELQQGLSQAKRDVQEQMAVLMQSREQVSEELVRLQKDNDSLQGKHSLHVSLQQAEDFILPDTTEALRELVLKYREDIINVRTAADHVEEKLKAEILFLKEQIQAEQCLKENLEETLQLEIENCKEEIASISSLKAELERIKVEKGQLESTLREKSQQLESLQEIKISLEEQLKKETAAKATVEQLMFEEKNKAQRLQTELDVSEQVQRDFVKLSQTLQVQLERIRQADSLERIRAILNDTKLTDINQLPET",
# RAB35_HUMAN
"MARDYDHLFKLLIIGDSGVGKSSLLLRFADNTFSGSYITTIGVDFKIRTVEINGEKVKLQIWDTAGQERFRTITSTYYRGTHGVIVVYDVTSAESFVNVKRWLHEINQNCDDVCRILVGNKNDDPERKVVETEDAYKFAGQMGIQLFETSAKENVNVEEMFNCITELVLRAKKDNLAKQQQQQQNDVVKLTKNSKRKKRCC",
# ERBB4_HUMAN
"MKPATGLWVWVSLLVAAGTVQPSDSQSVCAGTENKLSSLSDLEQQYRALRKYYENCEVVMGNLEITSIEHNRDLSFLRSVREVTGYVLVALNQFRYLPLENLRIIRGTKLYEDRYALAIFLNYRKDGNFGLQELGLKNLTEILNGGVYVDQNKFLCYADTIHWQDIVRNPWPSNLTLVSTNGSSGCGRCHKSCTGRCWGPTENHCQTLTRTVCAEQCDGRCYGPYVSDCCHRECAGGCSGPKDTDCFACMNFNDSGACVTQCPQTFVYNPTTFQLEHNFNAKYTYGAFCVKKCPHNFVVDSSSCVRACPSSKMEVEENGIKMCKPCTDICPKACDGIGTGSLMSAQTVDSSNIDKFINCTKINGNLIFLVTGIHGDPYNAIEAIDPEKLNVFRTVREITGFLNIQSWPPNMTDFSVFSNLVTIGGRVLYSGLSLLILKQQGITSLQFQSLKEISAGNIYITDNSNLCYYHTINWTTLFSTINQRIVIRDNRKAENCTAEGMVCNHLCSSDGCWGPGPDQCLSCRRFSRGRICIESCNLYDGEFREFENGSICVECDPQCEKMEDGLLTCHGPGPDNCTKCSHFKDGPNCVEKCPDGLQGANSFIFKYADPDRECHPCHPNCTQGCNGPTSHDCIYYPWTGHSTLPQHARTPLIAAGVIGGLFILVIVGLTFAVYVRRKSIKKKRALRRFLETELVEPLTPSGTAPNQAQLRILKETELKRVKVLGSGAFGTVYKGIWVPEGETVKIPVAIKILNETTGPKANVEFMDEALIMASMDHPHLVRLLGVCLSPTIQLVTQLMPHGCLLEYVHEHKDNIGSQLLLNWCVQIAKGMMYLEERRLVHRDLAARNVLVKSPNHVKITDFGLARLLEGDEKEYNADGGKMPIKWMALECIHYRKFTHQSDVWSYGVTIWELMTFGGKPYDGIPTREIPDLLEKGERLPQPPICTIDVYMVMVKCWMIDADSRPKFKELAAEFSRMARDPQRYLVIQGDDRMKLPSPNDSKFFQNLLDEEDLEDMMDAEEYLVPQAFNIPPPIYTSRARIDSNRSEIGHSPPPAYTPMSGNQFVYRDGGFAAEQGVSVPYRAPTSTIPEAPVAQGATAEIFDDSCCNGTLRKPVAPHVQEDSSTQRYSADPTVFAPERSPRGELDEEGYMTPMRDKPKQEYLNPVEENPFVSRRKNGDLQALDNPEYHNASNGPPKAEDEYVNEPLYLNTFANTLGKAEYLKNNILSMPEKAKKAFDNPDYWNHSLPPRSTLQHPDYLQEYSTKYFYKQNGRIRPIVAENPEYLSEFSLKPGTVLPPPPYRHRNTVV",
# RBP1_HUMAN
"MTECFLPPTSSPSEHRRVEHGSGLTRTPSSEEISPTKFPGLYRTGEPSPPHDILHEPPDVVSDDEKDHGKKKGKFKKKEKRTEGYAAFQEDSSGDEAESPSKMKRSKGIHVFKKPSFSKKKEKDFKIKEKPKEEKHKEEKHKEEKHKEKKSKDLTAADVVKQWKEKKKKKKPIQEPEVPQIDVPNLKPIFGIPLADAVERTMMYDGIRLPAVFRECIDYVEKYGMKCEGIYRVSGIKSKVDELKAAYDREESTNLEDYEPNTVASLLKQYLRDLPENLLTKELMPRFEEACGRTTETEKVQEFQRLLKELPECNYLLISWLIVHMDHVIAKELETKMNIQNISIVLSPTVQISNRVLYVFFTHVQELFGNVVLKQVMKPLRWSNMATMPTLPETQAGIKEEIRRQEFLLNCLHRDLQGGIKDLSKEERLWEVQRILTALKRKLREAKRQECETKIAQEIASLSKEDVSKEEMNENEEVINILLAQENEILTEQEELLAMEQFLRRQIASEKEEIERLRAEIAEIQSRQQHGRSETEEYSSESESESEDEEELQIILEDLQRQNEELEIKNNHLNQAIHEEREAIIELRVQLRLLQMQRAKAEQQAQEDEEPEWRGGAVQPPRDGVLEPKAAKEQPKAGKEPAKPSPSRDRKETSI",
# L2GL1_HUMAN
"MMKFRFRRQGADPQREKLKQELFAFNKTVEHGFPNQPSALAFDPELRIMAIGTRSGAVKIYGAPGVEFTGLHRDAATVTQMHFLTGQGRLLSLLDDSSLHLWEIVHHNGCAHLEEALSFQLPSRPGFDGASAPLSLTRVTVVLLVAASDIAALGTEGSSVFFLDVTTLTLLEGQTLAPGEVLRSVPDDYRCGKALGPVESLQGHLRDPTKILIGYSRGLLVIWNQASQCVDHIFLGNQQLESLCWGRDSSTVVSSHSDGSYAVWSVDAGSFPTLQPTVATTPYGPFPCKAINKILWRNCESGGHFIIFSGGMPRASYGDRHCVSVLRAETLVTLDFTSRIIDFFTVHSTRPEDEFDDPQALAVLLEEELVVLDLQTPGWPAVPAPYLAPLHSSAITCSAHVASVPAKLWARIVSAGEQQSPQPVSSALSWPITGGRNLAQEPSQRGLLLTGHEDGTVRFWDASGVALRPLYKLSTAGLFQTDCEHADSLAQAAEDDWPPFRKVGCFDPYSDDPRLGVQKVALCKYTAQMVVAGTAGQVLVLELSDVPVEQAVSVAIIDLLQDREGFTWKGHERLSPRTGPLPWPAGFQPRVLVQCLPPAAVTAVTLHTEWSLVAFGTSHGFGLFDYQRKSPVLARCTLHPNDSLAMEGPLSRVKSLKKSLRQSFRRIRKSRVSGKKRAANASSKLQEANAQLAEQACPHDVEMTPVQRRIEPRSADDSLSGVVRCLYFADTFLRDGAHHGPTMWAGTNSGSVFAYALEVPAAAVGGEKRPEQAVEAVLGKEVQLMHRAPVVAIAVLDGRGRPLPEPYEASRDLAQAPDMQGGHAVLIASEEQFKVFTLPKVSAKTKFKLTAHEGCRVRKVALATFASVACEDYAETCLACLTNLGDVHVFSVPGLRPQVHYSCIRKEDISGIASCVFTRHGQGFYLISPSEFERFSLSARNITEPLCSLDINWPRDATQASYRIRESPKLSQANGTPSILLAPQSLDGSPDPAHSMGPDTPEPPEAALSPMSIDSATSADTTLDTTGDVTVEDVKDFLGSSEESEKNLRNLAEDEAHACAILIK",
# PCBP1_HUMAN
"MDAGVTESGLNVTLTIRLLMHGKEVGSIIGKKGESVKRIREESGARINISEGNCPERIITLTGPTNAIFKAFAMIIDKLEEDINSSMTNSTAASRPPVTLRLVVPATQCGSLIGKGGCKIKEIRESTGAQVQVAGDMLPNSTERAITIAGVPQSVTECVKQICLVMLETLSQSPQGRVMTIPYQPMPASSPVICAGGQDRCSDAAGYPHATHDLEGPPLDAYSIQGQHTISPLDLAKLNQVARQQSHFAMMHGGTGFAGIDSSSPEVKGYWASLDASTQTTHELTIPNNLIGCIIGRQGANINEIRQMSGAQIKIANPVEGSSGRQVTITGSAASISLAQYLINARLSSEKGMGCS",
# PCBP2_HUMAN
"MDTGVIEGGLNVTLTIRLLMHGKEVGSIIGKKGESVKKMREESGARINISEGNCPERIITLAGPTNAIFKAFAMIIDKLEEDISSSMTNSTAASRPPVTLRLVVPASQCGSLIGKGGCKIKEIRESTGAQVQVAGDMLPNSTERAITIAGIPQSIIECVKQICVVMLETLSQSPPKGVTIPYRPKPSSSPVIFAGGQDRYSTGSDSASFPHTTPSMCLNPDLEGPPLEAYTIQGQYAIPQPDLTKLHQLAMQQSHFPMTHGNTGFSGIESSSPEVKGYWGLDASAQTTSHELTIPNDLIGCIIGRQGAKINEIRQMSGAQIKIANPVEGSTDRQVTITGSAASISLAQYLINVRLSSETGGMGSS",
# EPHA7_HUMAN
"MVFQTRYPSWIILCYIWLLRFAHTGEAQAAKEVLLLDSKAQQTELEWISSPPNGWEEISGLDENYTPIRTYQVCQVMEPNQNNWLRTNWISKGNAQRIFVELKFTLRDCNSLPGVLGTCKETFNLYYYETDYDTGRNIRENLYVKIDTIAADESFTQGDLGERKMKLNTEVREIGPLSKKGFYLAFQDVGACIALVSVKVYYKKCWSIIENLAIFPDTVTGSEFSSLVEVRGTCVSSAEEEAENAPRMHCSAEGEWLVPIGKCICKAGYQQKGDTCEPCGRGFYKSSSQDLQCSRCPTHSFSDKEGSSRCECEDGYYRAPSDPPYVACTRPPSAPQNLIFNINQTTVSLEWSPPADNGGRNDVTYRILCKRCSWEQGECVPCGSNIGYMPQQTGLEDNYVTVMDLLAHANYTFEVEAVNGVSDLSRSQRLFAAVSITTGQAAPSQVSGVMKERVLQRSVELSWQEPEHPNGVITEYEIKYYEKDQRERTYSTVKTKSTSASINNLKPGTVYVFQIRAFTAAGYGNYSPRLDVATLEEATGKMFEATAVSSEQNPVIIIAVVAVAGTIILVFMVFGFIIGRRHCGYSKADQEGDEELYFHFKFPGTKTYIDPETYEDPNRAVHQFAKELDASCIKIERVIGAGEFGEVCSGRLKLPGKRDVAVAIKTLKVGYTEKQRRDFLCEASIMGQFDHPNVVHLEGVVTRGKPVMIVIEFMENGALDAFLRKHDGQFTVIQLVGMLRGIAAGMRYLADMGYVHRDLAARNILVNSNLVCKVSDFGLSRVIEDDPEAVYTTTGGKIPVRWTAPEAIQYRKFTSASDVWSYGIVMWEVMSYGERPYWDMSNQDVIKAIEEGYRLPAPMDCPAGLHQLMLDCWQKERAERPKFEQIVGILDKMIRNPNSLKTPLGTCSRPISPLLDQNTPDFTTFCSVGEWLQAIKMERYKDNFTAAGYNSLESVARMTIEDVMSLGITLVGHQKKIMSSIQTMRAQMLHLHGTGIQV",
# RHEB_HUMAN
"MPQSKSRKIAILGYRSVGKSSLTIQFVEGQFVDSYDPTIENTFTKLITVNGQEYHLQLVDTAGQDEYSIFPQTYSIDINGYILVYSVTSIKSFEVIKVIHGKLLDMVGKVQIPIMLVGNKKDLHMERVISYEEGKALAESWNAAFLESSAKENQTAVDVFRRIILEAEKMDGAASQGKSSCSVM",
# CNN3_HUMAN
"MTHFNKGPSYGLSAEVKNKIASKYDHQAEEDLRNWIEEVTGMSIGPNFQLGLKDGIILCELINKLQPGSVKKVNESSLNWPQLENIGNFIKAIQAYGMKPHDIFEANDLFENGNMTQVQTTLVALAGLAKTKGFHTTIDIGVKYAEKQTRRFDEGKLKAGQSVIGLQMGTNKCASQAGMTAYGTRRHLYDPKMQTDKPFDQTTISLQMGTNKGASQAGMLAPGTRRDIYDQKLTLQPVDNSTISLQMGTNKVASQKGMSVYGLGRQVYDPKYCAAPTEPVIHNGSQGTGTNGSEISDSDYQAEYPDEYHGEYQDDYPRDYQYSDQGIDY",
# KS6A1_HUMAN
"MPLAQLKEPWPLMELVPLDPENGQTSGEEAGLQPSKDEGVLKEISITHHVKAGSEKADPSHFELLKVLGQGSFGKVFLVRKVTRPDSGHLYAMKVLKKATLKVRDRVRTKMERDILADVNHPFVVKLHYAFQTEGKLYLILDFLRGGDLFTRLSKEVMFTEEDVKFYLAELALGLDHLHSLGIIYRDLKPENILLDEEGHIKLTDFGLSKEAIDHEKKAYSFCGTVEYMAPEVVNRQGHSHSADWWSYGVLMFEMLTGSLPFQGKDRKETMTLILKAKLGMPQFLSTEAQSLLRALFKRNPANRLGSGPDGAEEIKRHVFYSTIDWNKLYRREIKPPFKPAVAQPDDTFYFDTEFTSRTPKDSPGIPPSAGAHQLFRGFSFVATGLMEDDGKPRAPQAPLHSVVQQLHGKNLVFSDGYVVKETIGVGSYSECKRCVHKATNMEYAVKVIDKSKRDPSEEIEILLRYGQHPNIITLKDVYDDGKHVYLVTELMRGGELLDKILRQKFFSEREASFVLHTIGKTVEYLHSQGVVHRDLKPSNILYVDESGNPECLRICDFGFAKQLRAENGLLMTPCYTANFVAPEVLKRQGYDEGCDIWSLGILLYTMLAGYTPFANGPSDTPEEILTRIGSGKFTLSGGNWNTVSETAKDLVSKMLHVDPHQRLTAKQVLQHPWVTQKDKLPQSQLSHQDLQLVKGAMAATYSALNSSKPTPQLKPIESSILAQRRVRKLPSTTL",
# CYH1_HUMAN
"MEEDDSYVPSDLTAEERQELENIRRRKQELLADIQRLKDEIAEVANEIENLGSTEERKNMQRNKQVAMGRKKFNMDPKKGIQFLIENDLLKNTCEDIAQFLYKGEGLNKTAIGDYLGERDEFNIQVLHAFVELHEFTDLNLVQALRQFLWSFRLPGEAQKIDRMMEAFAQRYCQCNNGVFQSTDTCYVLSFAIIMLNTSLHNPNVKDKPTVERFIAMNRGINDGGDLPEELLRNLYESIKNEPFKIPEDDGNDLTHTFFNPDREGWLLKLGGGRVKTWKRRWFILTDNCLYYFEYTTDKEPRGIIPLENLSIREVEDSKKPNCFELYIPDNKDQVIKACKTEADGRVVEGNHTVYRISAPTPEEKEEWIKCIKAAISRDPFYEMLAARKKKVSSTKRH",
# SHH_HUMAN
"MLLLARCLLLVLVSSLLVCSGLACGPGRGFGKRRHPKKLTPLAYKQFIPNVAEKTLGASGRYEGKISRNSERFKELTPNYNPDIIFKDEENTGADRLMTQRCKDKLNALAISVMNQWPGVKLRVTEGWDEDGHHSEESLHYEGRAVDITTSDRDRSKYGMLARLAVEAGFDWVYYESKAHIHCSVKAENSVAAKSGGCFPGSATVHLEQGGTKLVKDLSPGDRVLAADDQGRLLYSDFLTFLDRDDGAKKVFYVIETREPRERLLLTAAHLLFVAPHNDSATGEPEASSGSGPPSGGALGPRALFASRVRPGQRVYVVAERDGDRRLLPAAVHSVTLSEEAAGAYAPLTAQGTILINRVLASCYAVIEEHSWAHRAFAPFRLAHALLAALAPARTDRGGDSGGGDRGGGGGRVALTAPGAADAPGAGATAGIHWYSQLLYQIGTWLLDSEALHPLGMAVKSS",
# TERF2_HUMAN
"MAAGAGTAGPASGPGVVRDPAASQPRKRPGREGGEGARRSDTMAGGGGSSDGSGRAAGRRASRSSGRARRGRHEPGLGGPAERGAGEARLEEAVNRWVLKFYFHEALRAFRGSRYGDFRQIRDIMQALLVRPLGKEHTVSRLLRVMQCLSRIEEGENLDCSFDMEAELTPLESAINVLEMIKTEFTLTEAVVESSRKLVKEAAVIICIKNKEFEKASKILKKHMSKDPTTQKLRNDLLNIIREKNLAHPVIQNFSYETFQQKMLRFLESHLDDAEPYLLTMAKKALKSESAASSTGKEDKQPAPGPVEKPPREPARQLRNPPTTIGMMTLKAAFKTLSGAQDSEAAFAKLDQKDLVLPTQALPASPALKNKRPRKDENESSAPADGEGGSELQPKNKRMTISRLVLEEDSQSTEPSAGLNSSQEAASAPPSKPTVLNQPLPGEKNPKVPKGKWNSSNGVEEKETWVEEDELFQVQAAPDEDSTTNITKKQKWTVEESEWVKAGVQKYGEGNWAAISKNYPFVNRTAVMIKDRWRTMKRLGMN",
# DIXC1_HUMAN
"MLACLTRGNLLDVLQEGFNEQQLQAYVAWVNAQLKKRPAVKPVQDLRQDLRDGVILAYLIEIVAGEKLSGVQLSPGNQQEMKNNVEKVLQFVASKKIRMHQTSAKDIVDGNLKSIMRLVLALAAHFKPGSSRTVNQGRDSRAPLQSHRPHCATAVAQGAAAALADVCHDMSRSGRDVFRYRQRNSSMDEEIENPYWSVRALVQQYEGQQRSPSESSCSSLTSPSPIHSAKSESIITQSEEKADFVIIPAEGIENRTEGTDSPLSRDWRPGSPGTYLETSWEEQLLEQQEYLEKEMEEAKKMISGLQALLLNGSLPEDEQERPLALCEPGVNPEEQLIIIQSRLDQSMEENQDLKKELLKCKQEARNLQGIKDALQQRLTQQDTSVLQLKQELLRANMDKDELHNQNVDLQRKLDERNRLLGEYKKELGQKDRLLQQHQAKLEEALRKLSDVSYHQVDLERELEHKDVLLAHCMKREADEATNYNSHNSQSNGFLLPTAGKGATSVSNRGTSDLQLVRDALRSLRNSFSGHDPQHHTIDSLEQGISSLMERLHVMETQKKQERKVRVKSPRTQVGSEYRESWPPNSKLPHSQSSPTVSSTCTKVLYFTDRSLTPFMVNIPKRLEEVTLKDFKAAIDREGNHRYHFKALDPEFGTVKEEIFHDDDAIPGWEGKIVAWVEEDHGEN",
# DLG2_HUMAN
"MFFACYCALRTNVKKYRYQDEDAPHDHSLPRLTHEVRGPELVHVSEKNLSQIENVHGYVLQSHISPLKASPAPIIVNTDTLDTIPYVNGTEIEYEFEEITLERGNSGLGFSIAGGTDNPHIGDDPGIFITKIIPGGAAAEDGRLRVNDCILRVNEVDVSEVSHSKAVEALKEAGSIVRLYVRRRRPILETVVEIKLFKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIDGGAAQKDGRLQVGDRLLMVNNYSLEEVTHEEAVAILKNTSEVVYLKVGKPTTIYMTDPYGPPDITHSYSPPMENHLLSGNNGTLEYKTSLPPISPGRYSPIPKHMLVDDDYTRPPEPVYSTVNKLCDKPASPRHYSPVECDKSFLLSAPYSHYHLGLLPDSEMTSHSQHSTATRQPSMTLQRAVSLEGEPRKVVLHKGSTGLGFNIVGGEDGEGIFVSFILAGGPADLSGELQRGDQILSVNGIDLRGASHEQAAAALKGAGQTVTIIAQYQPEDYARFEAKIHDLREQMMNHSMSSGSGSLRTNQKRSLYVRAMFDYDKSKDSGLPSQGLSFKYGDILHVINASDDEWWQARRVMLEGDSEEMGVIPSKRRVERKERARLKTVKFNAKPGVIDSKGSFNDKRKKSFIFSRKFPFYKNKEQSEQETSDPERGQEDLILSYEPVTRQEINYTRPVIILGPMKDRINDDLISEFPDKFGSCVPHTTRPKRDYEVDGRDYHFVISREQMEKDIQEHKFIEAGQYNDNLYGTSVQSVRFVAERGKHCILDVSGNAIKRLQVAQLYPIAIFIKPRSLEPLMEMNKRLTEEQAKKTYDRAIKLEQEFGEYFTAIVQGDTLEDIYNQCKLVIEEQSGPFIWIPSKEKL",
# ELAV1_HUMAN
"MSNGYEDHMAEDCRGDIGRTNLIVNYLPQNMTQDELRSLFSSIGEVESAKLIRDKVAGHSLGYGFVNYVTAKDAERAINTLNGLRLQSKTIKVSYARPSSEVIKDANLYISGLPRTMTQKDVEDMFSRFGRIINSRVLVDQTTGLSRGVAFIRFDKRSEAEEAITSFNGHKPPGSSEPITVKFAANPNQNKNVALLSQLYHSPARRFGGPVHHQAQRFRFSPMGVDHMSGLSGVNVPGNASSGWCIFIYNLGQDADEGILWQMFGPFGAVTNVKVIRDFNTNKCKGFGFVTMTNYEEAAMAIASLNGYRLGDKILQVSFKTNKSHK",
# MYLK_HUMAN
"MGDVKLVASSHISKTSLSVDPSRVDSMPLTEAPAFILPPRNLCIKEGATAKFEGRVRGYPEPQVTWHRNGQPITSGGRFLLDCGIRGTFSLVIHAVHEEDRGKYTCEATNGSGARQVTVELTVEGSFAKQLGQPVVSKTLGDRFSAPAVETRPSIWGECPPKFATKLGRVVVKEGQMGRFSCKITGRPQPQVTWLKGNVPLQPSARVSVSEKNGMQVLEIHGVNQDDVGVYTCLVVNGSGKASMSAELSIQGLDSANRSFVRETKATNSDVRKEVTNVISKESKLDSLEAAAKSKNCSSPQRGGSPPWAANSQPQPPRESKLESCKDSPRTAPQTPVLQKTSSSITLQAARVQPEPRAPGLGVLSPSGEERKRPAPPRPATFPTRQPGLGSQDVVSKAANRRIPMEGQRDSAFPKFESKPQSQEVKENQTVKFRCEVSGIPKPEVAWFLEGTPVRRQEGSIEVYEDAGSHYLCLLKARTRDSGTYSCTASNAQGQLSCSWTLQVERLAVMEVAPSFSSVLKDCAVIEGQDFVLQCSVRGTPVPRITWLLNGQPIQYARSTCEAGVAELHIQDALPEDHGTYTCLAENALGQVSCSAWVTVHEKKSSRKSEYLLPVAPSKPTAPIFLQGLSDLKVMDGSQVTMTVQVSGNPPPEVIWLHNGNEIQESEDFHFEQRGTQHSLCIQEVFPEDTGTYTCEAWNSAGEVRTQAVLTVQEPHDGTQPWFISKPRSVTASLGQSVLISCAIAGDPFPTVHWLRDGKALCKDTGHFEVLQNEDVFTLVLKKVQPWHAGQYEILLKNRVGECSCQVSLMLQNSSARALPRGREPASCEDLCGGGVGADGGGSDRYGSLRPGWPARGQGWLEEEDGEDVRGVLKRRVETRQHTEEAIRQQEVEQLDFRDLLGKKVSTKTLSEDDLKEIPAEQMDFRANLQRQVKPKTVSEEERKVHSPQQVDFRSVLAKKGTSKTPVPEKVPPPKPATPDFRSVLGGKKKLPAENGSSSAETLNAKAVESSKPLSNAQPSGPLKPVGNAKPAETLKPMGNAKPAETLKPMGNAKPDENLKSASKEELKKDVKNDVNCKRGHAGTTDNEKRSESQGTAPAFKQKLQDVHVAEGKKLLLQCQVSSDPPATIIWTLNGKTLKTTKFIILSQEGSLCSVSIEKALPEDRGLYKCVAKNDAGQAECSCQVTVDDAPASENTKAPEMKSRRPKSSLPPVLGTESDATVKKKPAPKTPPKAAMPPQIIQFPEDQKVRAGESVELFGKVTGTQPITCTWMKFRKQIQESEHMKVENSENGSKLTILAARQEHCGCYTLLVENKLGSRQAQVNLTVVDKPDPPAGTPCASDIRSSSLTLSWYGSSYDGGSAVQSYSIEIWDSANKTWKELATCRSTSFNVQDLLPDHEYKFRVRAINVYGTSEPSQESELTTVGEKPEEPKDEVEVSDDDEKEPEVDYRTVTINTEQKVSDFYDIEERLGSGKFGQVFRLVEKKTRKVWAGKFFKAYSAKEKENIRQEISIMNCLHHPKLVQCVDAFEEKANIVMVLEIVSGGELFERIIDEDFELTERECIKYMRQISEGVEYIHKQGIVHLDLKPENIMCVNKTGTRIKLIDFGLARRLENAGSLKVLFGTPEFVAPEVINYEPIGYATDMWSIGVICYILVSGLSPFMGDNDNETLANVTSATWDFDDEAFDEISDDAKDFISNLLKKDMKNRLDCTQCLQHPWLMKDTKNMEAKKLSKDRMKKYMARRKWQKTGNAVRAIGRLSSMAMISGLSGRKSSTGSPTSPLNAEKLESEEDVSQAFLEAVAEEKPHVKPYFSKTIRDLEVVEGSAARFDCKIEGYPDPEVVWFKDDQSIRESRHFQIDYDEDGNCSLIISDVCGDDDAKYTCKAVNSLGEATCTAELIVETMEEGEGEGEEEEE",
# NPY5R_HUMAN
"MDLELDEYYNKTLATENNTAATRNSDFPVWDDYKSSVDDLQYFLIGLYTFVSLLGFMGNLLILMALMKKRNQKTTVNFLIGNLAFSDILVVLFCSPFTLTSVLLDQWMFGKVMCHIMPFLQCVSVLVSTLILISIAIVRYHMIKHPISNNLTANHGYFLIATVWTLGFAICSPLPVFHSLVELQETFGSALLSSRYLCVESWPSDSYRIAFTISLLLVQYILPLVCLTVSHTSVCRSISCGLSNKENRLEENEMINLTLHPSKKSGPQVKLSGSHKWSYSFIKKHRRRYSKKTACVLPAPERPSQENHSRILPENFGSVRSQLSSSSKFIPGVPTCFEIKPEENSDVHELRVKRSVTRIKKRSRSVFYRLTILILVFAVSWMPLHLFHVVTDFNDNLISNRHFKLVYCICHLLGMMSCCLNPILYGFLNNGIKADLVSLIHCLHM",
# EFNB3_HUMAN
"MGPPHSGPGGVRVGALLLLGVLGLVSGLSLEPVYWNSANKRFQAEGGYVLYPQIGDRLDLLCPRARPPGPHSSPNYEFYKLYLVGGAQGRRCEAPPAPNLLLTCDRPDLDLRFTIKFQEYSPNLWGHEFRSHHDYYIIATSDGTREGLESLQGGVCLTRGMKVLLRVGQSPRGGAVPRKPVSEMPMERDRGAAHSLEPGKENLPGDPTSNATSRGAEGPLPPPSMPAVAGAAGGLALLLLGVAGAGGAMCWRRRRAKPSESRHPGPGSFGRGGSLGLGGGGGMGPREAEPGELGIALRGGGAADPPFCPHYEKVSGDYGHPVYIVQDGPPQSPPNIYYKV",
# MLF2_HUMAN
"MFRFMRDVEPEDPMFLMDPFAIHRQHMSRMLSGGFGYSPFLSITDGNMPGTRPASRRMQQAGAVSPFGMLGMSGGFMDMFGMMNDMIGNMEHMTAGGNCQTFSSSTVISYSNTGDGAPKVYQETSEMRSAPGGIRETRRTVRDSDSGLEQMSIGHHIRDRAHILQRSRNHRTGDQEERQDYINLDESEAAAFDDEWRRETSRFRQQRPLEFRRLESSGAGGRRAEGPPRLAIQGPEDSPSRQSRRYDW",
# ITSN1_HUMAN
"MAQFPTPFGGSLDIWAITVEERAKHDQQFHSLKPISGFITGDQARNFFFQSGLPQPVLAQIWALADMNNDGRMDQVEFSIAMKLIKLKLQGYQLPSALPPVMKQQPVAISSAPAFGMGGIASMPPLTAVAPVPMGSIPVVGMSPTLVSSVPTAAVPPLANGAPPVIQPLPAFAHPAATLPKSSSFSRSGPGSQLNTKLQKAQSFDVASVPPVAEWAVPQSSRLKYRQLFNSHDKTMSGHLTGPQARTILMQSSLPQAQLASIWNLSDIDQDGKLTAEEFILAMHLIDVAMSGQPLPPVLPPEYIPPSFRRVRSGSGISVISSTSVDQRLPEEPVLEDEQQQLEKKLPVTFEDKKRENFERGNLELEKRRQALLEQQRKEQERLAQLERAEQERKERERQEQERKRQLELEKQLEKQRELERQREEERRKEIERREAAKRELERQRQLEWERNRRQELLNQRNKEQEDIVVLKAKKKTLEFELEALNDKKHQLEGKLQDIRCRLTTQRQEIESTNKSRELRIAEITHLQQQLQESQQMLGRLIPEKQILNDQLKQVQQNSLHRDSLVTLKRALEAKELARQHLRDQLDEVEKETRSKLQEIDIFNNQLKELREIHNKQQLQKQKSMEAERLKQKEQERKIIELEKQKEEAQRRAQERDKQWLEHVQQEDEHQRPRKLHEEEKLKREESVKKKDGEEKGKQEAQDKLGRLFHQHQEPAKPAVQAPWSTAEKGPLTISAQENVKVVYYRALYPFESRSHDEITIQPGDIVMVKGEWVDESQTGEPGWLGGELKGKTGWFPANYAEKIPENEVPAPVKPVTDSTSAPAPKLALRETPAPLAVTSSEPSTTPNNWADFSSTWPTSTNEKPETDNWDAWAAQPSLTVPSAGQLRQRSAFTPATATGSSPSPVLGQGEKVEGLQAQALYPWRAKKDNHLNFNKNDVITVLEQQDMWWFGEVQGQKGWFPKSYVKLISGPIRKSTSMDSGSSESPASLKRVASPAAKPVVSGEEFIAMYTYESSEQGDLTFQQGDVILVTKKDGDWWTGTVGDKAGVFPSNYVRLKDSEGSGTAGKTGSLGKKPEIAQVIASYTATGPEQLTLAPGQLILIRKKNPGGWWEGELQARGKKRQIGWFPANYVKLLSPGTSKITPTEPPKSTALAAVCQVIGMYDYTAQNDDELAFNKGQIINVLNKEDPDWWKGEVNGQVGLFPSNYVKLTTDMDPSQQWCSDLHLLDMLTPTERKRQGYIHELIVTEENYVNDLQLVTEIFQKPLMESELLTEKEVAMIFVNWKELIMCNIKLLKALRVRKKMSGEKMPVKMIGDILSAQLPHMQPYIRFCSRQLNGAALIQQKTDEAPDFKEFVKRLAMDPRCKGMPLSSFILKPMQRVTRYPLIIKNILENTPENHPDHSHLKHALEKAEELCSQVNEGVREKENSDRLEWIQAHVQCEGLSEQLVFNSVTNCLGPRKFLHSGKLYKAKSNKELYGFLFNDFLLLTQITKPLGSSGTDKVFSPKSNLQYKMYKTPIFLNEVLVKLPTDPSGDEPIFHISHIDRVYTLRAESINERTAWVQKIKAASELYIETEKKKREKAYLVRSQRATGIGRLMVNVVEGIELKPCRSHGKSNPYCEVTMGSQCHITKTIQDTLNPKWNSNCQFFIRDLEQEVLCITVFERDQFSPDDFLGRTEIRVADIKKDQGSKGPVTKCLLLHEVPTGEIVVRLDLQLFDEP",
# NPTX1_HUMAN
"MPAGRAARTCALLALCLLGAGAQDFGPTRFICTSVPVDADMCAASVAAGGAEELRSSVLQLRETVLQQKETILSQKETIRELTAKLGRCESQSTLDPGAGEARAGGGRKQPGSGKNTMGDLSRTPAAETLSQLGQTLQSLKTRLENLEQYSRLNSSSQTNSLKDLLQSKIDELERQVLSRVNTLEEGKGGPRNDTEERVKIETALTSLHQRISELEKGQKDNRPGDKFQLTFPLRTNYMYAKVKKSLPEMYAFTVCMWLKSSATPGVGTPFSYAVPGQANELVLIEWGNNPMEILINDKVAKLPFVINDGKWHHICVTWTTRDGVWEAYQDGTQGGSGENLAPYHPIKPQGVLVLGQEQDTLGGGFDATQAFVGELAHFNIWDRKLTPGEVYNLATCSTKALSGNVIAWAESHIEIYGGATKWTFEACRQIN",
# ACHA6_HUMAN
"MLTSKGQGFLHGGLCLWLCVFTPFFKGCVGCATEERLFHKLFSHYNQFIRPVENVSDPVTVHFEVAITQLANVDEVNQIMETNLWLRHIWNDYKLRWDPMEYDGIETLRVPADKIWKPDIVLYNNAVGDFQVEGKTKALLKYNGMITWTPPAIFKSSCPMDITFFPFDHQNCSLKFGSWTYDKAEIDLLIIGSKVDMNDFWENSEWEIIDASGYKHDIKYNCCEEIYTDITYSFYIRRLPMFYTINLIIPCLFISFLTVLVFYLPSDCGEKVTLCISVLLSLTVFLLVITETIPSTSLVVPLVGEYLLFTMIFVTLSIVVTVFVLNIHYRTPTTHTMPRWVKTVFLKLLPQVLLMRWPLDKTRGTGSDAVPRGLARRPAKGKLASHGEPRHLKECFHCHKSNELATSKRRLSHQPLQWVVENSEHSPEVEDVINSVQFIAENMKSHNETKEVEDDWKYVAMVVDRVFLWVFIIVCVFGTAGLFLQPLLGNTGKS",
# VAMP3_HUMAN
"MSTGPTAATGSNRRLQQTQNQVDEVVDIMRVNVDKVLERDQKLSELDDRADALQAGASQFETSAAKLKRKYWWKNCKMWAIGITVLVIFIIIIIVWVVSS",
# KCNJ8_HUMAN
"MLARKSIIPEEYVLARIAAENLRKPRIRDRLPKARFIAKSGACNLAHKNIREQGRFLQDIFTTLVDLKWRHTLVIFTMSFLCSWLLFAIMWWLVAFAHGDIYAYMEKSGMEKSGLESTVCVTNVRSFTSAFLFSIEVQVTIGFGGRMMTEECPLAITVLILQNIVGLIINAVMLGCIFMKTAQAHRRAETLIFSRHAVIAVRNGKLCFMFRVGDLRKSMIISASVRIQVVKKTTTPEGEVVPIHQLDIPVDNPIESNNIFLVAPLIICHVIDKRSPLYDISATDLANQDLEVIVILEGVVETTGITTQARTSYIAEEIQWGHRFVSIVTEEEGVYSVDYSKFGNTVKVAAPRCSARELDEKPSILIQTLQKSELSHQNSLRKRNSMRRNNSMRRNNSIRRNNSSLMVPKVQFMTPEGNQNTSES",
# NEDD8_HUMAN
"MLIKVKTLTGKEIEIDIEPTDKVERIKERVEEKEGIPPQQQRLIYSGKQMNDEKTAADYKILGGSVLHLVLALRGGGGLRQ",
# CAC1E_HUMAN
"MARFGEAVVARPGSGDGDSDQSRNRQGTPVPASGQAAAYKQTKAQRARTMALYNPIPVRQNCFTVNRSLFIFGEDNIVRKYAKKLIDWPPFEYMILATIIANCIVLALEQHLPEDDKTPMSRRLEKTEPYFIGIFCFEAGIKIVALGFIFHKGSYLRNGWNVMDFIVVLSGILATAGTHFNTHVDLRTLRAVRVLRPLKLVSGIPSLQIVLKSIMKAMVPLLQIGLLLFFAILMFAIIGLEFYSGKLHRACFMNNSGILEGFDPPHPCGVQGCPAGYECKDWIGPNDGITQFDNILFAVLTVFQCITMEGWTTVLYNTNDALGATWNWLYFIPLIIIGSFFVLNLVLGVLSGEFAKERERVENRRAFMKLRRQQQIERELNGYRAWIDKAEEVMLAEENKNAGTSALEVLRRATIKRSRTEAMTRDSSDEHCVDISSVGTPLARASIKSAKVDGVSYFRHKERLLRISIRHMVKSQVFYWIVLSLVALNTACVAIVHHNQPQWLTHLLYYAEFLFLGLFLLEMSLKMYGMGPRLYFHSSFNCFDFGVTVGSIFEVVWAIFRPGTSFGISVLRALRLLRIFKITKYWASLRNLVVSLMSSMKSIISLLFLLFLFIVVFALLGMQLFGGRFNFNDGTPSANFDTFPAAIMTVFQILTGEDWNEVMYNGIRSQGGVSSGMWSAIYFIVLTLFGNYTLLNVFLAIAVDNLANAQELTKDEQEEEEAFNQKHALQKAKEVSPMSAPNMPSIERDRRRRHHMSMWEPRSSHLRERRRRHHMSVWEQRTSQLRKHMQMSSQEALNREEAPTMNPLNPLNPLSSLNPLNAHPSLYRRPRAIEGLALGLALEKFEEERISRGGSLKGDGGDRSSALDNQRTPLSLGQREPPWLARPCHGNCDPTQQEAGGGEAVVTFEDRARHRQSQRRSRHRRVRTEGKESSSASRSRSASQERSLDEAMPTEGEKDHELRGNHGAKEPTIQEERAQDLRRTNSLMVSRGSGLAGGLDEADTPLVLPHPELEVGKHVVLTEQEPEGSSEQALLGNVQLDMGRVISQSEPDLSCITANTDKATTESTSVTVAIPDVDPLVDSTVVHISNKTDGEASPLKEAEIREDEEEVEKKKQKKEKRETGKAMVPHSSMFIFSTTNPIRRACHYIVNLRYFEMCILLVIAASSIALAAEDPVLTNSERNKVLRYFDYVFTGVFTFEMVIKMIDQGLILQDGSYFRDLWNILDFVVVVGALVAFALANALGTNKGRDIKTIKSLRVLRVLRPLKTIKRLPKLKAVFDCVVTSLKNVFNILIVYKLFMFIFAVIAVQLFKGKFFYCTDSSKDTEKECIGNYVDHEKNKMEVKGREWKRHEFHYDNIIWALLTLFTVSTGEGWPQVLQHSVDVTEEDRGPSRSNRMEMSIFYVVYFVVFPFFFVNIFVALIIITFQEQGDKMMEECSLEKNERACIDFAISAKPLTRYMPQNRHTFQYRVWHFVVSPSFEYTIMAMIALNTVVLMMKYYSAPCTYELALKYLNIAFTMVFSLECVLKVIAFGFLNYFRDTWNIFDFITVIGSITEIILTDSKLVNTSGFNMSFLKLFRAARLIKLLRQGYTIRILLWTFVQSFKALPYVCLLIAMLFFIYAIIGMQVFGNIKLDEESHINRHNNFRSFFGSLMLLFRSATGEAWQEIMLSCLGEKGCEPDTTAPSGQNENERCGTDLAYVYFVSFIFFCSFLMLNLFVAVIMDNFEYLTRDSSILGPHHLDEFVRVWAEYDRAACGRIHYTEMYEMLTLMSPPLGLGKRCPSKVAYKRLVLMNMPVAEDMTVHFTSTLMALIRTALDIKIAKGGADRQQLDSELQKETLAIWPHLSQKMLDLLVPMPKASDLTVGKIYAAMMIMDYYKQSKVKKQRQQLEEQKNAPMFQRMEPSSLPQEIIANAKALPYLQQDPVSGLSGRSGYPSMSPLSPQDIFQLACMDPADDGQFQERQSLEPEVSELKSVQPSNHGIYLPSDTQEHAGSGRASSMPRLTVDPQVVTDPSSMRRSFSTIRDKRSNSSWLEEFSMERSSENTYKSRRRSYHSSLRLSAHRLNSDSGHKSDTHRSGGRERGRSKERKHLLSPDVSRCNSEERGTQADWESPERRQSRSPSEGRSQTPNRQGTGSLSESSIPSVSDTSTPRRSRRQLPPVPPKPRPLLSYSSLIRHAGSISPPADGSEEGSPLTSQALESNNACLTESSNSPHPQQSQHASPQRYISEPYLALHEDSHASDCGEEETLTFEAAVATSLGRSNTIGSAPPLRHSWQMPNGHYRRRRRGGPGPGMMCGAVNNLLSDTEEDDKC",
# VAS1_HUMAN
"MMAAMATARVRMGPRCAQALWRMPWLPVFLSLAAAAAAAAAEQQVPLVLWSSDRDLWAPAADTHEGHITSDLQLSTYLDPALELGPRNVLLFLQDKLSIEDFTAYGGVFGNKQDSAFSNLENALDLAPSSLVLPAVDWYAVSTLTTYLQEKLGASPLHVDLATLRELKLNASLPALLLIRLPYTASSGLMAPREVLTGNDEVIGQVLSTLKSEDVPYTAALTAVRPSRVARDVAVVAGGLGRQLLQKQPVSPVIHPPVSYNDTAPRILFWAQNFSVAYKDQWEDLTPLTFGVQELNLTGSFWNDSFARLSLTYERLFGTTVTFKFILANRLYPVSARHWFTMERLEVHSNGSVAYFNASQVTGPSIYSFHCEYVSSLSKKGSLLVARTQPSPWQMMLQDFQIQAFNVMGEQFSYASDCASFFSPGIWMGLLTSLFMLFIFTYGLHMILSLKTMDRFDDHKGPTISLTQIV",
# RB11B_HUMAN
"MGTRDDEYDYLFKVVLIGDSGVGKSNLLSRFTRNEFNLESKSTIGVEFATRSIQVDGKTIKAQIWDTAGQERYRAITSAYYRGAVGALLVYDIAKHLTYENVERWLKELRDHADSNIVIMLVGNKSDLRHLRAVPTDEARAFAEKNNLSFIETSALDSTNVEEAFKNILTEIYRIVSQKQIADRAAHDESPGNNVVDISVPPTTDGQKPNKLQCCQNL",
# GRIK4_HUMAN
"MPRVSAPLVLLPAWLVMVACSPHSLRIAAILDDPMECSRGERLSITLAKNRINRAPERLGKAKVEVDIFELLRDSEYETAETMCQILPKGVVAVLGPSSSPASSSIISNICGEKEVPHFKVAPEEFVKFQFQRFTTLNLHPSNTDISVAVAGILNFFNCTTACLICAKAECLLNLEKLLRQFLISKDTLSVRMLDDTRDPTPLLKEIRDDKTATIIIHANASMSHTILLKAAELGMVSAYYTYIFTNLEFSLQRMDSLVDDRVNILGFSIFNQSHAFFQEFAQSLNQSWQENCDHVPFTGPALSSALLFDAVYAVVTAVQELNRSQEIGVKPLSCGSAQIWQHGTSLMNYLRMVELEGLTGHIEFNSKGQRSNYALKILQFTRNGFRQIGQWHVAEGLSMDSHLYASNISDTLFNTTLVVTTILENPYLMLKGNHQEMEGNDRYEGFCVDMLKELAEILRFNYKIRLVGDGVYGVPEANGTWTGMVGELIARKADLAVAGLTITAEREKVIDFSKPFMTLGISILYRVHMGRKPGYFSFLDPFSPGVWLFMLLAYLAVSCVLFLVARLTPYEWYSPHPCAQGRCNLLVNQYSLGNSLWFPVGGFMQQGSTIAPRALSTRCVSGVWWAFTLIIISSYTANLAAFLTVQRMDVPIESVDDLADQTAIEYGTIHGGSSMTFFQNSRYQTYQRMWNYMYSKQPSVFVKSTEEGIARVLNSNYAFLLESTMNEYYRQRNCNLTQIGGLLDTKGYGIGMPVGSVFRDEFDLAILQLQENNRLEILKRKWWEGGKCPKEEDHRAKGLGMENIGGIFVVLICGLIVAIFMAMLEFLWTLRHSEATEVSVCQEMVTELRSIILCQDSIHPRRRRAAVPPPRPPIPEERRPRGTATLSNGKLCGAGEPDQLAQRLAQEAALVARGCTHIRVCPECRRFQGLRARPSPARSEESLEWEKTTNSSEPE",
# SYUB_HUMAN
"MDVFMKGLSMAKEGVVAAAEKTKQGVTEAAEKTKEGVLYVGSKTREGVVQGVASVAEKTKEQASHLGGAVFSGAGNIAAATGLVKREEFPTDLKPEEVAQEAAEEPLIEPLMEPEGESYEDPPQEEYQEYEPEA",
# SEPT7_HUMAN
"MSVSARSAAAEERSVNSSTMVAQQKNLEGYVGFANLPNQVYRKSVKRGFEFTLMVVGESGLGKSTLINSLFLTDLYSPEYPGPSHRIKKTVQVEQSKVLIKEGGVQLLLTIVDTPGFGDAVDNSNCWQPVIDYIDSKFEDYLNAESRVNRRQMPDNRVQCCLYFIAPSGHGLKPLDIEFMKRLHEKVNIIPLIAKADTLTPEECQQFKKQIMKEIQEHKIKIYEFPETDDEEENKLVKKIKDRLPLAVVGSNTIIEVNGKRVRGRQYPWGVAEVENGEHCDFTILRNMLIRTHMQDLKDVTNNVHYENYRSRKLAAVTYNGVDNNKNKGQLTKSPLAQMEEERREHVAKMKKMEMEMEQVFEMKVKEKVQKLKDSEAELQRRHEQMKKNLEAQHKELEEKRRQFEDEKANWEAQQRILEQQNSSRTLEKNKKKGKIF",
# NTRK3_HUMAN
"MDVSLCPAKCSFWRIFLLGSVWLDYVGSVLACPANCVCSKTEINCRRPDDGNLFPLLEGQDSGNSNGNASINITDISRNITSIHIENWRSLHTLNAVDMELYTGLQKLTIKNSGLRSIQPRAFAKNPHLRYINLSSNRLTTLSWQLFQTLSLRELQLEQNFFNCSCDIRWMQLWQEQGEAKLNSQNLYCINADGSQLPLFRMNISQCDLPEISVSHVNLTVREGDNAVITCNGSGSPLPDVDWIVTGLQSINTHQTNLNWTNVHAINLTLVNVTSEDNGFTLTCIAENVVGMSNASVALTVYYPPRVVSLEEPELRLEHCIEFVVRGNPPPTLHWLHNGQPLRESKIIHVEYYQEGEISEGCLLFNKPTHYNNGNYTLIAKNPLGTANQTINGHFLKEPFPESTDNFILFDEVSPTPPITVTHKPEEDTFGVSIAVGLAAFACVLLVVLFVMINKYGRRSKFGMKGPVAVISGEEDSASPLHHINHGITTPSSLDAGPDTVVIGMTRIPVIENPQYFRQGHNCHKPDTYVQHIKRRDIVLKRELGEGAFGKVFLAECYNLSPTKDKMLVAVKALKDPTLAARKDFQREAELLTNLQHEHIVKFYGVCGDGDPLIMVFEYMKHGDLNKFLRAHGPDAMILVDGQPRQAKGELGLSQMLHIASQIASGMVYLASQHFVHRDLATRNCLVGANLLVKIGDFGMSRDVYSTDYYRLFNPSGNDFCIWCEVGGHTMLPIRWMPPESIMYRKFTTESDVWSFGVILWEIFTYGKQPWFQLSNTEVIECITQGRVLERPRVCPKEVYDVMLGCWQREPQQRLNIKEIYKILHALGKATPIYLDILG",
# AINX_HUMAN
"MSFGSEHYLCSSSSYRKVFGDGSRLSARLSGAGGAGGFRSQSLSRSNVASSAACSSASSLGLGLAYRRPPASDGLDLSQAAARTNEYKIIRTNEKEQLQGLNDRFAVFIEKVHQLETQNRALEAELAALRQRHAEPSRVGELFQRELRDLRAQLEEASSARSQALLERDGLAEEVQRLRARCEEESRGREGAERALKAQQRDVDGATLARLDLEKKVESLLDELAFVRQVHDEEVAELLATLQASSQAAAEVDVTVAKPDLTSALREIRAQYESLAAKNLQSAEEWYKSKFANLNEQAARSTEAIRASREEIHEYRRQLQARTIEIEGLRGANESLERQILELEERHSAEVAGYQDSIGQLENDLRNTKSEMARHLREYQDLLNVKMALDIEIAAYRKLLEGEETRFSTSGLSISGLNPLPNPSYLLPPRILSATTSKVSSTGLSLKKEEEEEEASKVASKKTSQIGESFEEILEETVISTKKTEKSNIEETTISSQKI",
# LAMA4_HUMAN
"MALSSAWRSVLPLWLLWSAACSRAASGDDNAFPFDIEGSSAVGRQDPPETSEPRVALGRLPPAAEKCNAGFFHTLSGECVPCDCNGNSNECLDGSGYCVHCQRNTTGEHCEKCLDGYIGDSIRGAPQFCQPCPCPLPHLANFAESCYRKNGAVRCICNENYAGPNCERCAPGYYGNPLLIGSTCKKCDCSGNSDPNLIFEDCDEVTGQCRNCLRNTTGFKCERCAPGYYGDARIAKNCAVCNCGGGPCDSVTGECLEEGFEPPTGMDCPTISCDKCVWDLTDDLRLAALSIEEGKSGVLSVSSGAAAHRHVNEINATIYLLKTKLSERENQYALRKIQINNAENTMKSLLSDVEELVEKENQASRKGQLVQKESMDTINHASQLVEQAHDMRDKIQEINNKMLYYGEEHELSPKEISEKLVLAQKMLEEIRSRQPFFTQRELVDEEADEAYELLSQAESWQRLHNETRTLFPVVLEQLDDYNAKLSDLQEALDQALNYVRDAEDMNRATAARQRDHEKQQERVREQMEVVNMSLSTSADSLTTPRLTLSELDDIIKNASGIYAEIDGAKSELQVKLSNLSNLSHDLVQEAIDHAQDLQQEANELSRKLHSSDMNGLVQKALDASNVYENIVNYVSEANETAEFALNTTDRIYDAVSGIDTQIIYHKDESENLLNQARELQAKAESSSDEAVADTSRRVGGALARKSALKTRLSDAVKQLQAAERGDAQQRLGQSRLITEEANRTTMEVQQATAPMANNLTNWSQNLQHFDSSAYNTAVNSARDAVRNLTEVVPQLLDQLRTVEQKRPASNVSASIQRIRELIAQTRSVASKIQVSMMFDGQSAVEVHSRTSMDDLKAFTSLSLYMKPPVKRPELTETADQFILYLGSKNAKKEYMGLAIKNDNLVYVYNLGTKDVEIPLDSKPVSSWPAYFSIVKIERVGKHGKVFLTVPSLSSTAEEKFIKKGEFSGDDSLLDLDPEDTVFYVGGVPSNFKLPTSLNLPGFVGCLELATLNNDVISLYNFKHIYNMDPSTSVPCARDKLAFTQSRAASYFFDGSGYAVVRDITRRGKFGQVTRFDIEVRTPADNGLILLMVNGSMFFRLEMRNGYLHVFYDFGFSGGPVHLEDTLKKAQINDAKYHEISIIYHNDKKMILVVDRRHVKSMDNEKMKIPFTDIYIGGAPPEILQSRALRAHLPLDINFRGCMKGFQFQKKDFNLLEQTETLGVGYGCPEDSLISRRAYFNGQSFIASIQKISFFDGFEGGFNFRTLQPNGLLFYYASGSDVFSISLDNGTVIMDVKGIKVQSVDKQYNDGLSHFVISSVSPTRYELIVDKSRVGSKNPTKGKIEQTQASEKKFYFGGSPISAQYANFTGCISNAYFTRVDRDVEVEDFQRYTEKVHTSLYECPIESSPLFLLHKKGKNLSKPKASQNKKGGKSKDAPSWDPVALKLPERNTPRNSHCHLSNSPRAIEHAYQYGGTANSRQEFEHLKGDFGAKSQFSIRLRTRSSHGMIFYVSDQEENDFMTLFLAHGRLVYMFNVGHKKLKIRSQEKYNDGLWHDVIFIRERSSGRLVIDGLRVLEESLPPTEATWKIKGPIYLGGVAPGKAVKNVQINSIYSFSGCLSNLQLNGASITSASQTFSVTPCFEGPMETGTYFSTEGGYVVLDESFNIGLKFEIAFEVRPRSSSGTLVHGHSVNGEYLNVHMKNGQVIVKVNNGIRDFSTSVTPKQSLCDGRWHRITVIRDSNVVQLDVDSEVNHVVGPLNPKPIDHREPVFVGGVPESLLTPRLAPSKPFTGCIRHFVIDGHPVSFSKAALVSGAVSINSCPAA",
# GBRA6_HUMAN
"MASSLPWLCIILWLENALGKLEVEGNFYSENVSRILDNLLEGYDNRLRPGFGGAVTEVKTDIYVTSFGPVSDVEMEYTMDVFFRQTWTDERLKFGGPTEILSLNNLMVSKIWTPDTFFRNGKKSIAHNMTTPNKLFRIMQNGTILYTMRLTINADCPMRLVNFPMDGHACPLKFGSYAYPKSEIIYTWKKGPLYSVEVPEESSSLLQYDLIGQTVSSETIKSNTGEYVIMTVYFHLQRKMGYFMIQIYTPCIMTVILSQVSFWINKESVPARTVFGITTVLTMTTLSISARHSLPKVSYATAMDWFIAVCFAFVFSALIEFAAVNYFTNLQTQKAKRKAQFAAPPTVTISKATEPLEAEIVLHPDSKYHLKKRITSLSLPIVSSSEANKVLTRAPILQSTPVTPPPLSPAFGGTSKIDQYSRILFPVAFAGFNLVYWVVYLSKDTMEVSSSVE",
# GRIK5_HUMAN
"MPAELLLLLIVAFASPSCQVLSSLRMAAILDDQTVCGRGERLALALAREQINGIIEVPAKARVEVDIFELQRDSQYETTDTMCQILPKGVVSVLGPSSSPASASTVSHICGEKEIPHIKVGPEETPRLQYLRFASVSLYPSNEDVSLAVSRILKSFNYPSASLICAKAECLLRLEELVRGFLISKETLSVRMLDDSRDPTPLLKEIRDDKVSTIIIDANASISHLILRKASELGMTSAFYKYILTTMDFPILHLDGIVEDSSNILGFSMFNTSHPFYPEFVRSLNMSWRENCEASTYLGPALSAALMFDAVHVVVSAVRELNRSQEIGVKPLACTSANIWPHGTSLMNYLRMVEYDGLTGRVEFNSKGQRTNYTLRILEKSRQGHREIGVWYSNRTLAMNATTLDINLSQTLANKTLVVTTILENPYVMRRPNFQALSGNERFEGFCVDMLRELAELLRFRYRLRLVEDGLYGAPEPNGSWTGMVGELINRKADLAVAAFTITAEREKVIDFSKPFMTLGISILYRVHMGRKPGYFSFLDPFSPAVWLFMLLAYLAVSCVLFLAARLSPYEWYNPHPCLRARPHILENQYTLGNSLWFPVGGFMQQGSEIMPRALSTRCVSGVWWAFTLIIISSYTANLAAFLTVQRMEVPVESADDLADQTNIEYGTIHAGSTMTFFQNSRYQTYQRMWNYMQSKQPSVFVKSTEEGIARVLNSRYAFLLESTMNEYHRRLNCNLTQIGGLLDTKGYGIGMPLGSPFRDEITLAILQLQENNRLEILKRKWWEGGRCPKEEDHRAKGLGMENIGGIFIVLICGLIIAVFVAVMEFIWSTRRSAESEEVSVCQEMLQELRHAVSCRKTSRSRRRRRPGGPSRALLSLRAVREMRLSNGKLYSAGAGGDAGSAHGGPQRLLDDPGPPSGARPAAPTPCTHVRVCQECRRIQALRASGAGAPPRGLGVPAEATSPPRPRPGPAGPRELAEHE",
# PKN1_HUMAN
"MASDAVQSEPRSWSLLEQLGLAGADLAAPGVQQQLELERERLRREIRKELKLKEGAENLRRATTDLGRSLGPVELLLRGSSRRLDLLHQQLQELHAHVVLPDPAATHDGPQSPGAGGPTCSATNLSRVAGLEKQLAIELKVKQGAENMIQTYSNGSTKDRKLLLTAQQMLQDSKTKIDIIRMQLRRALQAGQLENQAAPDDTQGSPDLGAVELRIEELRHHFRVEHAVAEGAKNVLRLLSAAKAPDRKAVSEAQEKLTESNQKLGLLREALERRLGELPADHPKGRLLREELAAASSAAFSTRLAGPFPATHYSTLCKPAPLTGTLEVRVVGCRDLPETIPWNPTPSMGGPGTPDSRPPFLSRPARGLYSRSGSLSGRSSLKAEAENTSEVSTVLKLDNTVVGQTSWKPCGPNAWDQSFTLELERARELELAVFWRDQRGLCALKFLKLEDFLDNERHEVQLDMEPQGCLVAEVTFRNPVIERIPRLRRQKKIFSKQQGKAFQRARQMNIDVATWVRLLRRLIPNATGTGTFSPGASPGSEARTTGDISVEKLNLGTDSDSSPQKSSRDPPSSPSSLSSPIQESTAPELPSETQETPGPALCSPLRKSPLTLEDFKFLAVLGRGHFGKVLLSEFRPSGELFAIKALKKGDIVARDEVESLMCEKRILAAVTSAGHPFLVNLFGCFQTPEHVCFVMEYSAGGDLMLHIHSDVFSEPRAIFYSACVVLGLQFLHEHKIVYRDLKLDNLLLDTEGYVKIADFGLCKEGMGYGDRTSTFCGTPEFLAPEVLTDTSYTRAVDWWGLGVLLYEMLVGESPFPGDDEEEVFDSIVNDEVRYPRFLSAEAIGIMRRLLRRNPERRLGSSERDAEDVKKQPFFRTLGWEALLARRLPPPFVPTLSGRTDVSNFDEEFTGEAPTLSPPRDARPLTAAEQAAFLDFDFVAGGC",
# ASIC2_HUMAN
"MDLKESPSEGSLQPSSIQIFANTSTLHGIRHIFVYGPLTIRRVLWAVAFVGSLGLLLVESSERVSYYFSYQHVTKVDEVVAQSLVFPAVTLCNLNGFRFSRLTTNDLYHAGELLALLDVNLQIPDPHLADPSVLEALRQKANFKHYKPKQFSMLEFLHRVGHDLKDMMLYCKFKGQECGHQDFTTVFTKYGKCYMFNSGEDGKPLLTTVKGGTGNGLEIMLDIQQDEYLPIWGETEETTFEAGVKVQIHSQSEPPFIQELGFGVAPGFQTFVATQEQRLTYLPPPWGECRSSEMGLDFFPVYSITACRIDCETRYIVENCNCRMVHMPGDAPFCTPEQHKECAEPALGLLAEKDSNYCLCRTPCNLTRYNKELSMVKIPSKTSAKYLEKKFNKSEKYISENILVLDIFFEALNYETIEQKKAYEVAALLGDIGGQMGLFIGASILTILELFDYIYELIKEKLLDLLGKEEDEGSHDENVSTCDTMPNHSETISHTVNVPLQTTLGTLEEIAC",
# MK14_HUMAN
"MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKTGLRVAVKKLSRPFQSIIHAKRTYRELRLLKHMKHENVIGLLDVFTPARSLEEFNDVYLVTHLMGADLNNIVKCQKLTDDHVQFLIYQILRGLKYIHSADIIHRDLKPSNLAVNEDCELKILDFGLARHTDDEMTGYVATRWYRAPEIMLNWMHYNQTVDIWSVGCIMAELLTGRTLFPGTDHIDQLKLILRLVGTPGAELLKKISSESARNYIQSLTQMPKMNFANVFIGANPLAVDLLEKMLVLDSDKRITAAQALAHAYFAQYHDPDDEPVADPYDQSFESRDLLIDEWKSLTYDEVISFVPPPLDQEEMES",
# DPYL2_HUMAN
"MSYQGKKNIPRITSDRLLIKGGKIVNDDQSFYADIYMEDGLIKQIGENLIVPGGVKTIEAHSRMVIPGGIDVHTRFQMPDQGMTSADDFFQGTKAALAGGTTMIIDHVVPEPGTSLLAAFDQWREWADSKSCCDYSLHVDISEWHKGIQEEMEALVKDHGVNSFLVYMAFKDRFQLTDCQIYEVLSVIRDIGAIAQVHAENGDIIAEEQQRILDLGITGPEGHVLSRPEEVEAEAVNRAITIANQTNCPLYITKVMSKSSAEVIAQARKKGTVVYGEPITASLGTDGSHYWSKNWAKAAAFVTSPPLSPDPTTPDFLNSLLSCGDLQVTGSAHCTFNTAQKAVGKDNFTLIPEGTNGTEERMSVIWDKAVVTGKMDENQFVAVTSTNAAKVFNLYPRKGRIAVGSDADLVIWDPDSVKTISAKTHNSSLEYNIFEGMECRGSPLVVISQGKIVLEDGTLHVTEGSGRYIPRKPFPDFVYKRIKARSRLAELRGVPRGLYDGPVCEVSVTPKTVTPASSAKTSPAKQQAPPVRNLHQSGFSLSGAQIDDNIPRRTTQRIVAPPGGRANITSLG",
# SYPL1_HUMAN
"MAPNIYLVRQRISRLGQRMSGFQINLNPLKEPLGFIKVLEWIASIFAFATCGGFKGQTEIQVNCPPAVTENKTVTATFGYPFRLNEASFQPPPGVNICDVNWKDYVLIGDYSSSAQFYVTFAVFVFLYCIAALLLYVGYTSLYLDSRKLPMIDFVVTLVATFLWLVSTSAWAKALTDIKIATGHNIIDELPPCKKKAVLCYFGSVTSMGSLNVSVIFGFLNMILWGGNAWFVYKETSLHSPSNTSAPHSQGGIPPPTGI",
# KCC4_HUMAN
"MLKVTVPSCSASSCSSVTASAAPGTASLVPDYWIDGSNRDALSDFFEVESELGRGATSIVYRCKQKGTQKPYALKVLKKTVDKKIVRTEIGVLLRLSHPNIIKLKEIFETPTEISLVLELVTGGELFDRIVEKGYYSERDAADAVKQILEAVAYLHENGIVHRDLKPENLLYATPAPDAPLKIADFGLSKIVEHQVLMKTVCGTPGYCAPEILRGCAYGPEVDMWSVGIITYILLCGFEPFYDERGDQFMFRRILNCEYYFISPWWDEVSLNAKDLVRKLIVLDPKKRLTTFQALQHPWVTGKAANFVHMDTAQKKLQEFNARRKLKAAVKAVVASSRLGSASSSHGSIQESHKASRDPSPIQDGNEDMKAIPEGEKIQGDGAQAAVKGAQAELMKVQALEKVKGADINAEEAPKMVPKAVEDGIKVADLELEEGLAEEKLKTVEEAAAPREGQGSSAVGFEVPQQDVILPEY",
# VACHT_HUMAN
"MESAEPAGQARAAATKLSEAVGAALQEPRRQRRLVLVIVCVALLLDNMLYMVIVPIVPDYIAHMRGGGEGPTRTPEVWEPTLPLPTPANASAYTANTSASPTAAWPAGSALRPRYPTESEDVKIGVLFASKAILQLLVNPLSGPFIDRMSYDVPLLIGLGVMFASTVLFAFAEDYATLFAARSLQGLGSAFADTSGIAMIADKYPEEPERSRALGVALAFISFGSLVAPPFGGILYEFAGKRVPFLVLAAVSLFDALLLLAVAKPFSAAARARANLPVGTPIHRLMLDPYIAVVAGALTTCNIPLAFLEPTIATWMKHTMAASEWEMGMAWLPAFVPHVLGVYLTVRLAARYPHLQWLYGALGLAVIGASSCIVPACRSFAPLVVSLCGLCFGIALVDTALLPTLAFLVDVRHVSVYGSVYAIADISYSVAYALGPIVAGHIVHSLGFEQLSLGMGLANLLYAPVLLLLRNVGLLTRSRSERDVLLDEPPQGLYDAVRLRERPVSGQDGEPRSPPGPFDACEDDYNYYYTRS",
# CALRL_HUMAN
"MEKKCTLYFLVLLPFFMILVTAELEESPEDSIQLGVTRNKIMTAQYECYQKIMQDPIQQAEGVYCNRTWDGWLCWNDVAAGTESMQLCPDYFQDFDPSEKVTKICDQDGNWFRHPASNRTWTNYTQCNVNTHEKVKTALNLFYLTIIGHGLSIASLLISLGIFFYFKSLSCQRITLHKNLFFSFVCNSVVTIIHLTAVANNQALVATNPVSCKVSQFIHLYLMGCNYFWMLCEGIYLHTLIVVAVFAEKQHLMWYYFLGWGFPLIPACIHAIARSLYYNDNCWISSDTHLLYIIHGPICAALLVNLFFLLNIVRVLITKLKVTHQAESNLYMKAVRATLILVPLLGIEFVLIPWRPEGKIAEEVYDYIMHILMHFQGLLVSTIFCFFNGEVQAILRRNWNQYKIQFGNSFSNSEALRSASYTVSTISDGPGYSHDCPSEHLNGKSIHDIENVLLKPENLYN",
# NTRK2_HUMAN
"MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCSDPSPGIVAFPRLEPNSVDPENITEIFIANQKRLEIINEDDVEAYVGLRNLTIVDSGLKFVAHKAFLKNSNLQHINFTRNKLTSLSRKHFRHLDLSELILVGNPFTCSCDIMWIKTLQEAKSSPDTQDLYCLNESSKNIPLANLQIPNCGLPSANLAAPNLTVEEGKSITLSCSVAGDPVPNMYWDVGNLVSKHMNETSHTQGSLRITNISSDDSGKQISCVAENLVGEDQDSVNLTVHFAPTITFLESPTSDHHWCIPFTVKGNPKPALQWFYNGAILNESKYICTKIHVTNHTEYHGCLQLDNPTHMNNGDYTLIAKNEYGKDEKQISAHFMGWPGIDDGANPNYPDVIYEDYGTAANDIGDTTNRSNEIPSTDVTDKTGREHLSVYAVVVIASVVGFCLLVMLFLLKLARHSKFGMKGPASVISNDDDSASPLHHISNGSNTPSSSEGGPDAVIIGMTKIPVIENPQYFGITNSQLKPDTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLCPEQDKILVAVKTLKDASDNARKDFHREAELLTNLQHEHIVKFYGVCVEGDPLIMVFEYMKHGDLNKFLRAHGPDAVLMAEGNPPTELTQSQMLHIAQQIAAGMVYLASQHFVHRDLATRNCLVGENLLVKIGDFGMSRDVYSTDYYRVGGHTMLPIRWMPPESIMYRKFTTESDVWSLGVVLWEIFTYGKQPWYQLSNNEVIECITQGRVLQRPRTCPQEVYELMLGCWQREPHMRKNIKGIHTLLQNLAKASPVYLDILG",
# STX1A_HUMAN
"MKDRTQELRTAKDSDDDDDVAVTVDRDRFMDEFFEQVEEIRGFIDKIAENVEEVKRKHSAILASPNPDEKTKEELEELMSDIKKTANKVRSKLKSIEQSIEQEEGLNRSSADLRIRKTQHSTLSRKFVEVMSEYNATQSDYRERCKGRIQRQLEITGRTTTSEELEDMLESGNPAIFASGIIMDSSISKQALSEIETRHSEIIKLENSIRELHDMFMDMAMLVESQGEMIDRIEYNVEHAVDYVERAVSDTKKAVKYQSKARRKKIMIIICCVILGIVIASTVGGIFA",
# DREB_HUMAN
"MAGVSFSGHRLELLAAYEEVIREESAADWALYTYEDGSDDLKLAASGEGGLQELSGHFENQKVMYGFCSVKDSQAALPKYVLINWVGEDVPDARKCACASHVAKVAEFFQGVDVIVNASSVEDIDAGAIGQRLSNGLARLSSPVLHRLRLREDENAEPVGTTYQKTDAAVEMKRINREQFWEQAKKEEELRKEEERKKALDERLRFEQERMEQERQEQEERERRYREREQQIEEHRRKQQTLEAEEAKRRLKEQSIFGDHRDEEEETHMKKSESEVEEAAAIIAQRPDNPREFFKQQERVASASAGSCDVPSPFNHRPGSHLDSHRRMAPTPIPTRSPSDSSTASTPVAEQIERALDEVTSSQPPPLPPPPPPAQETQEPSPILDSEETRAAAPQAWAGPMEEPPQAQAPPRGPGSPAEDLMFMESAEQAVLAAPVEPATADATEIHDAADTIETDTATADTTVANNVPPAATSLIDLWPGNGEGASTLQGEPRAPTPPSGTEVTLAEVPLLDEVAPEPLLPAGEGCATLLNFDELPEPPATFCDPEEVEGESLAAPQTPTLPSALEELEQEQEPEPHLLTNGETTQKEGTQASEGYFSQSQEEEFAQSEELCAKAPPPVFYNKPPEIDITCWDADPVPEEEEGFEGGD",
# AT2B3_HUMAN
"MGDMANSSIEFHPKPQQQRDVPQAGGFGCTLAELRTLMELRGAEALQKIEEAYGDVSGLCRRLKTSPTEGLADNTNDLEKRRQIYGQNFIPPKQPKTFLQLVWEALQDVTLIILEVAAIVSLGLSFYAPPGEESEACGNVSGGAEDEGEAEAGWIEGAAILLSVICVVLVTAFNDWSKEKQFRGLQSRIEQEQKFTVIRNGQLLQVPVAALVVGDIAQVKYGDLLPADGVLIQANDLKIDESSLTGESDHVRKSADKDPMLLSGTHVMEGSGRMVVTAVGVNSQTGIIFTLLGAGGEEEEKKDKKGKQQDGAMESSQTKAKKQDGAVAMEMQPLKSAEGGEMEEREKKKANAPKKEKSVLQGKLTKLAVQIGKAGLVMSAITVIILVLYFVIETFVVEGRTWLAECTPVYVQYFVKFFIIGVTVLVVAVPEGLPLAVTISLAYSVKKMMKDNNLVRHLDACETMGNATAICSDKTGTLTTNRMTVVQSYLGDTHYKEIPAPSALTPKILDLLVHAISINSAYTTKILPPEKEGALPRQVGNKTECALLGFVLDLKRDFQPVREQIPEDKLYKVYTFNSVRKSMSTVIRMPDGGFRLFSKGASEILLKKCTNILNSNGELRGFRPRDRDDMVRKIIEPMACDGLRTICIAYRDFSAGQEPDWDNENEVVGDLTCIAVVGIEDPVRPEVPEAIRKCQRAGITVRMVTGDNINTARAIAAKCGIIQPGEDFLCLEGKEFNRRIRNEKGEIEQERLDKVWPKLRVLARSSPTDKHTLVKGIIDSTTGEQRQVVAVTGDGTNDGPALKKADVGFAMGIAGTDVAKEASDIILTDDNFTSIVKAVMWGRNVYDSISKFLQFQLTVNVVAVIVAFTGACITQDSPLKAVQMLWVNLIMDTFASLALATEPPTESLLLRKPYGRDKPLISRTMMKNILGHAVYQLAIIFTLLFVGELFFDIDSGRNAPLHSPPSEHYTIIFNTFVMMQLFNEINARKIHGERNVFDGIFSNPIFCTIVLGTFGIQIVIVQFGGKPFSCSPLSTEQWLWCLFVGVGELVWGQVIATIPTSQLKCLKEAGHGPGKDEMTDEELAEGEEEIDHAERELRRGQILWFRGLNRIQTQIRVVKAFRSSLYEGLEKPESKTSIHNFMATPEFLINDYTHNIPLIDDTDVDENEERLRAPPPPSPNQNNNAIDSGIYLTTHVTKSATSSVFSSSPGSPLHSVETSL",
# PTPRO_HUMAN
"MGHLPTGIHGARRLLPLLWLFVLFKNATAFHVTVQDDNNIVVSLEASDVISPASVYVVKITGESKNYFFEFEEFNSTLPPPVIFKASYHGLYYIITLVVVNGNVVTKPSRSITVLTKPLPVTSVSIYDYKPSPETGVLFEIHYPEKYNVFTRVNISYWEGKDFRTMLYKDFFKGKTVFNHWLPGMCYSNITFQLVSEATFNKSTLVEYSGVSHEPKQHRTAPYPPQNISVRIVNLNKNNWEEQSGNFPEESFMRSQDTIGKEKLFHFTEETPEIPSGNISSGWPDFNSSDYETTSQPYWWDSASAAPESEDEFVSVLPMEYENNSTLSETEKSTSGSFSFFPVQMILTWLPPKPPTAFDGFHIHIEREENFTEYLMVDEEAHEFVAELKEPGKYKLSVTTFSSSGSCETRKSQSAKSLSFYISPSGEWIEELTEKPQHVSVHVLSSTTALMSWTSSQENYNSTIVSVVSLTCQKQKESQRLEKQYCTQVNSSKPIIENLVPGAQYQVVIYLRKGPLIGPPSDPVTFAIVPTGIKDLMLYPLGPTAVVLSWTRPYLGVFRKYVVEMFYFNPATMTSEWTTYYEIAATVSLTASVRIANLLPAWYYNFRVTMVTWGDPELSCCDSSTISFITAPVAPEITSVEYFNSLLYISWTYGDDTTDLSHSRMLHWMVVAEGKKKIKKSVTRNVMTAILSLPPGDIYNLSVTACTERGSNTSMLRLVKLEPAPPKSLFAVNKTQTSVTLLWVEEGVADFFEVFCQQVGSSQKTKLQEPVAVSSHVVTISSLLPATAYNCSVTSFSHDSPSVPTFIAVSTMVTEMNPNVVVISVLAILSTLLIGLLLVTLIILRKKHLQMARECGAGTFVNFASLERDGKLPYNWRRSIFAFLTLLPSCLWTDYLLAFYINPWSKNGLKKRKLTNPVQLDDFDAYIKDMAKDSDYKFSLQFEELKLIGLDIPHFAADLPLNRCKNRYTNILPYDFSRVRLVSMNEEEGADYINANYIPGYNSPQEYIATQGPLPETRNDFWKMVLQQKSQIIVMLTQCNEKRRVKCDHYWPFTEEPIAYGDITVEMISEEEQDDWACRHFRINYADEMQDVMHFNYTAWPDHGVPTANAAESILQFVHMVRQQATKSKGPMIIHCSAGVGRTGTFIALDRLLQHIRDHEFVDILGLVSEMRSYRMSMVQTEEQYIFIHQCVQLMWMKKKQQFCISDVIYENVSKS",
# PTPRN_HUMAN
"MRRPRRPGGLGGSGGLRLLLCLLLLSSRPGGCSAVSAHGCLFDRRLCSHLEVCIQDGLFGQCQVGVGQARPLLQVTSPVLQRLQGVLRQLMSQGLSWHDDLTQYVISQEMERIPRLRPPEPRPRDRSGLAPKRPGPAGELLLQDIPTGSAPAAQHRLPQPPVGKGGAGASSSLSPLQAELLPPLLEHLLLPPQPPHPSLSYEPALLQPYLFHQFGSRDGSRVSEGSPGMVSVGPLPKAEAPALFSRTASKGIFGDHPGHSYGDLPGPSPAQLFQDSGLLYLAQELPAPSRARVPRLPEQGSSSRAEDSPEGYEKEGLGDRGEKPASPAVQPDAALQRLAAVLAGYGVELRQLTPEQLSTLLTLLQLLPKGAGRNPGGVVNVGADIKKTMEGPVEGRDTAELPARTSPMPGHPTASPTSSEVQQVPSPVSSEPPKAARPPVTPVLLEKKSPLGQSQPTVAGQPSARPAAEEYGYIVTDQKPLSLAAGVKLLEILAEHVHMSSGSFINISVVGPALTFRIRHNEQNLSLADVTQQAGLVKSELEAQTGLQILQTGVGQREEAAAVLPQTAHSTSPMRSVLLTLVALAGVAGLLVALAVALCVRQHARQQDKERLAALGPEGAHGDTTFEYQDLCRQHMATKSLFNRAEGPPEPSRVSSVSSQFSDAAQASPSSHSSTPSWCEEPAQANMDISTGHMILAYMEDHLRNRDRLAKEWQALCAYQAEPNTCATAQGEGNIKKNRHPDFLPYDHARIKLKVESSPSRSDYINASPIIEHDPRMPAYIATQGPLSHTIADFWQMVWESGCTVIVMLTPLVEDGVKQCDRYWPDEGASLYHVYEVNLVSEHIWCEDFLVRSFYLKNVQTQETRTLTQFHFLSWPAEGTPASTRPLLDFRRKVNKCYRGRSCPIIVHCSDGAGRTGTYILIDMVLNRMAKGVKEIDIAATLEHVRDQRPGLVRSKDQFEFALTAVAEEVNAILKALPQ",
# VATF_HUMAN
"MAGRGKLIAVIGDEDTVTGFLLGGIGELNKNRHPNFLVVEKDTTINEIEDTFRQFLNRDDIGIILINQYIAEMVRHALDAHQQSIPAVLEIPSKEHPYDAAKDSILRRARGMFTAEDLR",
# RHG44_HUMAN
"MKKQFNRMRQLANQTVGRAEKTEVLSEDLLQVEKRLELVKQVSHSTHKKLTACLQGQQGAEADKRSKKLPLTTLAQCLMEGSAILGDDTLLGKMLKLCGETEDKLAQELIHFELQVERDVIEPLFLLAEVEIPNIQKQRKHLAKLVLDMDSSRTRWQQTSKSSGLSSSLQPAGAKADALREEMEEAANRVEICRDQLSADMYSFVAKEIDYANYFQTLIEVQAEYHRKSLTLLQAVLPQIKAQQEAWVEKPSFGKPLEEHLTISGREIAFPIEACVTMLLECGMQEEGLFRVAPSASKLKKLKAALDCCVVDVQEYSADPHAIAGALKSYLRELPEPLMTFELYDEWIQASNVQEQDKKLQALWNACEKLPKANHNNIRYLIKFLSKLSEYQDVNKMTPSNMAIVLGPNLLWPQAEGNITEMMTTVSLQIVGIIEPIIQHADWFFPGEIEFNITGNYGSPVHVNHNANYSSMPSPDMDPADRRQPEQARRPLSVATDNMMLEFYKKDGLRKIQSMGVRVMDTNWVARRGSSAGRKVSCAPPSMQPPAPPAELAAPLPSPLPEQPLDSPAAPALSPSGLGLQPGPERTSTTKSKELSPGSAQKGSPGSSQGTACAGTQPGAQPGAQPGASPSPSQPPADQSPHTLRKVSKKLAPIPPKVPFGQPGAMADQSAGQPSPVSLSPTPPSTPSPYGLSYPQGYSLASGQLSPAAAPPLASPSVFTSTLSKSRPTPKPRQRPTLPPPQPPTVNLSASSPQSTEAPMLDGMSPGESMSTDLVHFDIPSIHIELGSTLRLSPLEHMRRHSVTDKRDSEEESESTAL",
# CPEB4_HUMAN
"MGDYGFGVLVQSNTGNKSAFPVRFHPHLQPPHHHQNATPSPAAFINNNTAANGSSAGSAWLFPAPATHNIQDEILGSEKAKSQQQEQQDPLEKQQLSPSPGQEAGILPETEKAKSEENQGDNSSENGNGKEKIRIESPVLTGFDYQEATGLGTSTQPLTSSASSLTGFSNWSAAIAPSSSTIINEDASFFHQGGVPAASANNGALLFQNFPHHVSPGFGGSFSPQIGPLSQHHPHHPHFQHHHSQHQQQRRSPASPHPPPFTHRNAAFNQLPHLANNLNKPPSPWSSYQSPSPTPSSSWSPGGGGYGGWGGSQGRDHRRGLNGGITPLNSISPLKKNFASNHIQLQKYARPSSAFAPKSWMEDSLNRADNIFPFPDRPRTFDMHSLESSLIDIMRAENDTIKGRLNYSYPGSDSSLLINARTYGRRRGQSSLFPMEDGFLDDGRGDQPLHSGLGSPHCFSHQNGERVERYSRKVFVGGLPPDIDEDEITASFRRFGPLIVDWPHKAESKSYFPPKGYAFLLFQDESSVQALIDACIEEDGKLYLCVSSPTIKDKPVQIRPWNLSDSDFVMDGSQPLDPRKTIFVGGVPRPLRAVELAMIMDRLYGGVCYAGIDTDPELKYPKGAGRVAFSNQQSYIAAISARFVQLQHGEIDKRVEVKPYVLDDQLCDECQGARCGGKFAPFFCANVTCLQYYCEYCWAAIHSRAGREFHKPLVKEGGDRPRHISFRWN",
# HNRL2_HUMAN
"MEVKRLKVTELRSELQRRGLDSRGLKVDLAQRLQEALDAEMLEDEAGGGGAGPGGACKAEPRPVAASGGGPGGDEEEDEEEEEEDEEALLEDEDEEPPPAQALGQAAQPPPEPPEAAAMEAAAEPDASEKPAEATAGSGGVNGGEEQGLGKREEDEPEERSGDETPGSEVPGDKAAEEQGDDQDSEKSKPAGSDGERRGVKRQRDEKDEHGRAYYEFREEAYHSRSKSPLPPEEEAKDEEEDQTLVNLDTYTSDLHFQVSKDRYGGQPLFSEKFPTLWSGARSTYGVTKGKVCFEAKVTQNLPMKEGCTEVSLLRVGWSVDFSRPQLGEDEFSYGFDGRGLKAENGQFEEFGQTFGENDVIGCFANFETEEVELSFSKNGEDLGVAFWISKDSLADRALLPHVLCKNCVVELNFGQKEEPFFPPPEEFVFIHAVPVEERVRTAVPPKTIEECEVILMVGLPGSGKTQWALKYAKENPEKRYNVLGAETVLNQMRMKGLEEPEMDPKSRDLLVQQASQCLSKLVQIASRTKRNFILDQCNVYNSGQRRKLLLFKTFSRKVVVVVPNEEDWKKRLELRKEVEGDDVPESIMLEMKANFSLPEKCDYMDEVTYGELEKEEAQPIVTKYKEEARKLLPPSEKRTNRRNNRNKRNRQNRSRGQGYVGGQRRGYDNRAYGQQYWGQPGNRGGYRNFYDRYRGDYDRFYGRDYEYNRYRDYYRQYNRDWQSYYYHHPQDRDRYYRNYYGYQGYR",
# BLTP1_HUMAN
"MDQRKNESIVPSITQLEDFLTEHNSNVVWLLVATILSCGWIIYLTYYNSRNVGLILTLVLNRLYKHGYIHIGSFSFSVLSGKVMVREIYYITEDMSIRIQDGFIIFRWWKMYNPKQKQHDPKAETRLYITVNDFEFHVYNRSDLYGRLQELFGLEPTIIPPKKDDDKTREIGRTRTQSKIERVKVKTESQDPTSSWRSLIPVIKVNVSTGRLAFGNHYQPQTLCINFDDAFLTYTTKPPSSHLDQFMHIVKGKLENVRVMLVPSPRYVGLQNDEPPRLMGEGFVVMQSNDVDIYYYMDEPGLVPEETEENIEGEMSSEDCKLQDLPPCWGLDIVCGKGTDFNYGPWADRQRDCLWKFFFPPDYQVLKVSEIAQPGRPRQILAFELRMNIIADATIDLLFTKNRETNAVHVNVGAGSYLEINIPMTVEENGYTPAIKGQLLHVDATTSMQYRTLLEAEMLAFHINASYPRIWNMPQTWQCELEVYKATYHFIFAQKNFFTDLIQDWSSDSPPDIFSFVPYTWNFKIMFHQFEMIWAANQHNWIDCSTKQQENVYLAACGETLNIDFSLPFTDFVPATCNTKFSLRGEDVDLHLFLPDCHPSKYSLFMLVKNCHPNKMIHDTGIPAECQSGQKTVKPKWRNVTQEKSGWVECWTVPSVMLTIDYTWHPIYPQKADEQLKQSLSEMEETMLSVLRPSQKTSDRVVSSPSTSSRPPIDPSELPPDKLHVEMELSPDSQITLYGPLLNAFLCIKENYFGEDDMYMDFEEVISSPVLSLSTSSSSGWTAVGMENDKKENEGSAKSIHPLALRPWDITVLVNLYKVHGRLPVHGTTDGPECPTAFLERLCFEMKKGFRETMLQLILSPLNVFVSDNYQQRPPVDEVLREGHINLSGLQLRAHAMFSAEGLPLGSDSLEYAWLIDVQAGSLTAKVTAPQLACLLEWGQTFVFHVVCREYELERPKSVIICQHGIDRRFCESKLSCIPGPCPTSDDLKYTMIRLAVDGADIYIVEHGCATNIKMGAIRVANCNLHNQSVGEGISAAIQDFQVRQYIEQLNNCRIGLQPAVLRRAYWLEAGSANLGLITVDIALAADHHSKHEAQRHFLETHDARTKRLWFLWPDDILKNKRCRNKCGCLGGCRFFGGTVTGLDFFKLEELTPSSSSAFSSTSAESDMYYGQSLLQPGEWIITKEIPKIIDGNVNGMKRKEWENKSVGIEVERKTQHLSLQVPLRSHSSSSSSEENSSSSAAQPLLAGEKESPSSVADDHLVQKEFLHGTKRDDGQASIPTEISGNSPVSPNTQDKSVGQSPLRSPLKRQASVCSTRLGSTKSLTAAFYGDKQPVTVGVQFSSDVSRSDENVLDSPKQRRSFGSFPYTPSADSNSFHQYRSMDSSMSMADSEAYFSAAEEFEPISSDEGPGTYPGRKKKKKQTQQIDYSRGSIYHSVEGPLTGHGESIQDSRTLPFKTHPSQASFVSALGGEDDVIEHLYIVEGEKTVESEQITPQQPVMNCYQTYLTQFQVINWSVKHPTNKRTSKSSLHRPLDLDTPTSEESSSSFEQLSVPTFKVIKQGLTANSLLDRGMQLSGSTSNTPYTPLEKKLADNTDDETLTEEWTLDQPVSQTRTTAIVEVKGTVDIVLTPLVAEALDRYIEAMVHCASTRHPAAIVDDLHAKVLREAVQNSKTTFSENLSSKQDIRGTKTEQSTIGTTNQGQAQTNLTMKQDNVTIKGLQTNVSIPKVNLCLLQASVEESPTTAPSRSVTHVSLVALCFDRIATQVRMNRGVVEETSNNAEPGRTSNFDRYVHATKMQPQSSGSLRSNAGAEKGKEIAAKLNIHRVHGQLRGLDTTDIGTCAITAIPFEKSKVLFTLEELDEFTFVDETDQQAVPDVTRIGPSQEKWGWIMFECGLENLTIKGGRQSGAVLYNSFGIMGKASDTERGGVLTSNNSSDSPTGSGYNTDVSDDNLPCDRTSPSSDLNGNSVSDEQDEGVESDDLKKDLPLMPPPPDSCSMKLTIKEIWFSFAAPTNVRSHTHAFSRQLNLLSTATPAVGAWLVPIDQLKSSLNKLETEGTLRICAVMGCIMTEALENKSVHFPLRSKYNRLTKVARFLQENPSCLLCNILHHYLHQANYSIIDDATMSDGLPALVTLKKGLVALARQWMKFIVVTPAFKGVSLHRPAQPLKPQIAMDHEHEDGLGLDNGGGLQSDTSADGAEFEFDAATVSEHTMLLEGTANRPPPGSSGPVTGAEIMRKLSKTHTHSDSALKIKGIHPYHSLSYTSGDTATDSPVHVGRAGMPVKDSPRKESLLSYLTGSFPSLHNLLEGTPQRSSAAVKSSSLTRTGNTVATDMLSEHPLLSEPSSVSFYNWMSNAVGNRGSVLQESPVTKSGHNSLPTGVAPNLPTIPSASDFNTVLSSDQNTLDGTHSQHSTSQDDVAGVEEANQGFPAVQLADAQVVFKPLLSHTGIQSQDTMPFCYRMYFGEHLSFSGTLDCLRADIVDSDTAKERKGKRARRQGHVNLPPLEFKPALMLGTFSISAVVMEKSVCTPQNSTSALSFHDLSKRYYNTFHCNFTISCQSISQHVDMALVRLIHQFSTMIDDIKATQTDIKLSRYTAGSASPTPTFKTRKHRDFRSSDFSRSSRGSLNGGNRVNNAKNKRTNNENNKKESRNKNSLGRSERRTSKVSRKGSKDVVDHMTIHMDDSDSITVSEQSEPSAECWQNMYKLLNFYSLISDPTGILEKSSETFGPAGVRSPTEPTCKVVFENEQDNSSLTKTQRKRSLVTSEPQHVTLIVFGIGMVNRTHLEADIGGLTMESELKRIHGSFTLKEKMKDVLHQKMTETCATAHIGGVNIVLLEGITPNIQLEDFPTSPTSTAKQEFLTVVKCSIAKSQALYSAQRGLKTNNAAVFKVGAISINIPQHPATLHSMMVRSSHQLSKQISDLIRQPSTAPQPVKEDIATPLPSEKTPTSVNQTPVETNEFPQLPEGLEKKPIVLKFSAMLDGIAIGAALLPSLKAEYKMGRMRSHGMTGAQTRFTFELPNHRLRFTSKVSATDMSTIPPSASLNLPPVTMSGKYIMEEHDSYSDQVWSIDELPSKQGYYLQGNYLRCVAEVGSFEHNLTTDLLNHLVFVQKVFMKEVNEVIQKVSGGEQPIPLWNEHDGTADGDKPKILLYSLNLQFKGIQVTATTPSMRAVRFETGLIELELSNRLQTKASPGSSSYLKLFGKCQVDLNLALGQIVKHQVYEEAGSDFHQVAYFKTRIGLRNALREEISGSSDREAVLITLNRPIVYAQPVAFDRAVLFWLNYKAAYDNWNEQRMALHKDIHMATKEVVDMLPGIQQTSAQAFGTLFLQLTVNDLGICLPITNTAQSNHTGDLDTGSALVLTIESTLITACSSESLVSKGHFKNFCIRFADGFETSWDDWKPEIHGDLVMNACVVPDGTYEVCSRTTGQAAAESSSAGTWTLNVLWKMCGIDVHMDPNIGKRLNALGNTLTTLTGEEDIDDIADLNSVNIADLSDEDEVDTMSPTIHTEATDYRRQAASASQPGELRGRKIMKRIVDIRELNEQAKVIDDLKKLGASEGTINQEIQRYQQLESVAVNDIRRDVRKKLRRSSMRAASLKDKWGLSYKPSYSRSKSISASGRPPLKRMERASSRVGETEELPEIRVDAASPGPRVTFNIQDTFPEETELDLLSVTIEGPSHYSSNSEGSCSVFSSPKTPGGFSPGIPFQTEEGRRDDSLSSTSEDSEKDEKDEDHERERFYIYRKPSHTSRKKATGFAAVHQLFTERWPTTPVNRSLSGTATERNIDFELDIRVEIDSGKCVLHPTTLLQEHDDISLRRSYDRSSRSLDQDSPSKKKKFQTNYASTTHLMTGKKVPSSLQTKPSDLETTVFYIPGVDVKLHYNSKTLKTESPNASRGSSLPRTLSKESKLYGMKDSATSPPSPPLPSTVQSKTNTLLPPQPPPIPAAKGKGSGGVKTAKLYAWVALQSLPEEMVISPCLLDFLEKALETIPITPVERNYTAVSSQDEDMGHFEIPDPMEESTTSLVSSSTSAYSSFPVDVVVYVRVQPSQIKFSCLPVSRVECMLKLPSLDLVFSSNRGELETLGTTYPAETLSPGGNATQSGTKTSASKTGIPGSSGLGSPLGRSRHSSSQSDLTSSSSSSSGLSFTACMSDFSLYVFHPYGAGKQKTAVSGLTPGSGGLGNVDEEPTSVTGRKDSLSINLEFVKVSLSRIRRSGGASFFESQSVSKSASKMDTTLINISAVCDIGSASFKYDMRRLSEILAFPRAWYRRSIARRLFLGDQTINLPTSGPGTPDSIEGVSQHLSPESSRKAYCKTWEQPSQSASFTHMPQSPNVFNEHMTNSTMSPGTVGQSLKSPASIRSRSVSDSSVPRRDSLSKTSTPFNKSNKAASQQGTPWETLVVFAINLKQLNVQMNMSNVMGNTTWTTSGLKSQGRLSVGSNRDREISMSVGLGRSQLDSKGGVVGGTIDVNALEMVAHISEHPNQQPSHKIQITMGSTEARVDYMGSSILMGIFSNADLKLQDEWKVNLYNTLDSSITDKSEIFVHGDLKWDIFQVMISRSTTPDLIKIGMKLQEFFTQQFDTSKRALSTWGPVPYLPPKTMTSNLEKSSQEQLLDAAHHRHWPGVLKVVSGCHISLFQIPLPEDGMQFGGSMSLHGNHMTLACFHGPNFRSKSWALFHLEEPNIAFWTEAQKIWEDGSSDHSTYIVQTLDFHLGHNTMVTKPCGALESPMATITKITRRRHENPPHGVASVKEWFNYVTATRNEELNLLRNVDANNTENSTTVKNSSLLSGFRGGSSYNHETETIFALPRMQLDFKSIHVQEPQEPSLQDASLKPKVECSVVTEFTDHICVTMDAELIMFLHDLVSAYLKEKEKAIFPPRILSTRPGQKSPIIIHDDNSSDKDREDSITYTTVDWRDFMCNTWHLEPTLRLISWTGRKIDPVGVDYILQKLGFHHARTTIPKWLQRGVMDPLDKVLSVLIKKLGTALQDEKEKKGKDKEEH",
# AAK1_HUMAN
"MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGEDSLLDCSLLSNPTTDLLEEFAPTAISAPVHKAAEDSNLISGFDVPEGSDKVAEDEFDPIPVLITKNPQGGHSRNSSGSSESSLPNLARSLLLVDQLIDL",
# INSY1_HUMAN
"MNIRGAPDLGQPSDDPSSGGERERIRQRMKMVIGQLEGILRELKEVAKELREVVSQIDKLTSDFDFELEPDDWTTATVSSTSSSDKAGMGGPFDLGHLDFMTADILSDSWEFCSFLDVSTPSDSVDGPESTRPGAGPDYRLMNGGTPIPNGPRVETPDSSSEEAFGAGPTVKSQLPQRTPGTRERVRFSDKVLYHALCCDDEEGDGEQEVEEEEVGLPPEPAHTEAHAGPHKPSPAPYKSRRSPLTSRHSGSTLAPEQTRRVTRNSSTQTVSDKSTQTVLPYTATRQKARGKN",
# VWC2_HUMAN
"MPSSTAMAVGALSSSLLVTCCLMVALCSPSIPLEKLAQAPEQPGQEKREHASRDGPGRVNELGRPARDEGGSGRDWKSKSGRGLAGREPWSKLKQAWVSQGGGAKAGDLQVRPRGDTPQAEALAAAAQDAIGPELAPTPEPPEEYVYPDYRGKGCVDESGFVYAIGEKFAPGPSACPCLCTEEGPLCAQPECPRLHPRCIHVDTSQCCPQCKERKNYCEFRGKTYQTLEEFVVSPCERCRCEANGEVLCTVSACPQTECVDPVYEPDQCCPICKNGPNCFAETAVIPAGREVKTDECTICHCTYEEGTWRIERQAMCTRHECRQM",
# S4A8_HUMAN
"MPAAGSNEPDGVLSYQRPDEEAVVDQGGTSTILNIHYEKEELEGHRTLYVGVRMPLGRQSHRHHRTHGQKHRRRGRGKGASQGEEGLEALAHDTPSQRVQFILGTEEDEEHVPHELFTELDEICMKEGEDAEWKETARWLKFEEDVEDGGERWSKPYVATLSLHSLFELRSCLINGTVLLDMHANSIEEISDLILDQQELSSDLNDSMRVKVREALLKKHHHQNEKKRNNLIPIVRSFAEVGKKQSDPHLMDKHGQTVSPQSVPTTNLEVKNGVNCEHSPVDLSKVDLHFMKKIPTGAEASNVLVGEVDILDRPIVAFVRLSPAVLLSGLTEVPIPTRFLFILLGPVGKGQQYHEIGRSMATIMTDEIFHDVAYKAKERDDLLAGIDEFLDQVTVLPPGEWDPSIRIEPPKNVPSQEKRKMPGVPNGNVCHIEQEPHGGHSGPELQRTGRLFGGLVLDIKRKAPWYWSDYRDALSLQCLASFLFLYCACMSPVITFGGLLGEATEGRISAIESLFGASMTGIAYSLFAGQALTILGSTGPVLVFEKILFKFCKDYALSYLSLRACIGLWTAFLCIVLVATDASSLVCYITRFTEEAFASLICIIFIYEAIEKLIHLAETYPIHMHSQLDHLSLYYCRCTLPENPNNHTLQYWKDHNIVTAEVHWANLTVSECQEMHGEFMGSACGHHGPYTPDVLFWSCILFFTTFILSSTLKTFKTSRYFPTRVRSMVSDFAVFLTIFTMVIIDFLIGVPSPKLQVPSVFKPTRDDRGWIINPIGPNPWWTVIAAIIPALLCTILIFMDQQITAVIINRKEHKLKKGCGYHLDLLMVAIMLGVCSIMGLPWFVAATVLSITHVNSLKLESECSAPGEQPKFLGIREQRVTGLMIFVLMGCSVFMTAILKFIPMPVLYGVFLYMGVSSLQGIQFFDRLKLFGMPAKHQPDFIYLRHVPLRKVHLFTLIQLTCLVLLWVIKASPAAIVFPMMVLALVFVRKVMDLCFSKRELSWLDDLMPESKKKKLDDAKKKAKEEEEAEKMLEIGGDKFPLESRKLLSSPGKNISCRCDPSEINISDEMPKTTVWKALSMNSGNAKEKSLFN",
# LEGL_HUMAN
"MAGSVADSDAVVKLDDGHLNNSLSSPVQADVYFPRLIVPFCGHIKGGMRPGKKVLVMGIVDLNPESFAISLTCGDSEDPPADVAIELKAVFTDRQLLRNSCISGERGEEQSAIPYFPFIPDQPFRVEILCEHPRFRVFVDGHQLFDFYHRIQTLSAIDTIKINGDLQITKLG",
# SV2C_HUMAN
"MEDSYKDRTSLMKGAKDIAREVKKQTVKKVNQAVDRAQDEYTQRSYSRFQDEEDDDDYYPAGETYNGEANDDEGSSEATEGHDEDDEIYEGEYQGIPSMNQAKDSIVSVGQPKGDEYKDRRELESERRADEEELAQQYELIIQECGHGRFQWALFFVLGMALMADGVEVFVVGFVLPSAETDLCIPNSGSGWLGSIVYLGMMVGAFFWGGLADKVGRKQSLLICMSVNGFFAFLSSFVQGYGFFLFCRLLSGFGIGGAIPTVFSYFAEVLAREKRGEHLSWLCMFWMIGGIYASAMAWAIIPHYGWSFSMGSAYQFHSWRVFVIVCALPCVSSVVALTFMPESPRFLLEVGKHDEAWMILKLIHDTNMRARGQPEKVFTVNKIKTPKQIDELIEIESDTGTWYRRCFVRIRTELYGIWLTFMRCFNYPVRDNTIKLTIVWFTLSFGYYGLSVWFPDVIKPLQSDEYALLTRNVERDKYANFTINFTMENQIHTGMEYDNGRFIGVKFKSVTFKDSVFKSCTFEDVTSVNTYFKNCTFIDTVFDNTDFEPYKFIDSEFKNCSFFHNKTGCQITFDDDYSAYWIYFVNFLGTLAVLPGNIVSALLMDRIGRLTMLGGSMVLSGISCFFLWFGTSESMMIGMLCLYNGLTISAWNSLDVVTVELYPTDRRATGFGFLNALCKAAAVLGNLIFGSLVSITKSIPILLASTVLVCGGLVGLCLPDTRTQVLM",
# VP26B_HUMAN
"MSFFGFGQSVEVEILLNDAESRKRAEHKTEDGKKEKYFLFYDGETVSGKVSLALKNPNKRLEHQGIKIEFIGQIELYYDRGNHHEFVSLVKDLARPGEITQSQAFDFEFTHVEKPYESYTGQNVKLRYFLRATISRRLNDVVKEMDIVVHTLSTYPELNSSIKMEVGIEDCLHIEFEYNKSKYHLKDVIVGKIYFLLVRIKIKHMEIDIIKRETTGTGPNVYHENDTIAKYEIMDGAPVRGESIPIRLFLAGYELTPTMRDINKKFSVRYYLNLVLIDEEERRYFKQQEVVLWRKGDIVRKSMSHQAAIASQRFEGTTSLGEVRTPSQLSDNNCRQ",
# ANO6_HUMAN
"MKKMSRNVLLQMEEEEDDDDGDIVLENLGQTIVPDLGSLESQHDFRTPEFEEFNGKPDSLFFNDGQRRIDFVLVYEDESRKETNKKGTNEKQRRKRQAYESNLICHGLQLEATRSVLDDKLVFVKVHAPWEVLCTYAEIMHIKLPLKPNDLKNRSSAFGTLNWFTKVLSVDESIIKPEQEFFTAPFEKNRMNDFYIVDRDAFFNPATRSRIVYFILSRVKYQVINNVSKFGINRLVNSGIYKAAFPLHDCKFRRQSEDPSCPNERYLLYREWAHPRSIYKKQPLDLIRKYYGEKIGIYFAWLGYYTQMLLLAAVVGVACFLYGYLNQDNCTWSKEVCHPDIGGKIIMCPQCDRLCPFWKLNITCESSKKLCIFDSFGTLVFAVFMGVWVTLFLEFWKRRQAELEYEWDTVELQQEEQARPEYEARCTHVVINEITQEEERIPFTAWGKCIRITLCASAVFFWILLIIASVIGIIVYRLSVFIVFSAKLPKNINGTDPIQKYLTPQTATSITASIISFIIIMILNTIYEKVAIMITNFELPRTQTDYENSLTMKMFLFQFVNYYSSCFYIAFFKGKFVGYPGDPVYWLGKYRNEECDPGGCLLELTTQLTIIMGGKAIWNNIQEVLLPWIMNLIGRFHRVSGSEKITPRWEQDYHLQPMGKLGLFYEYLEMIIQFGFVTLFVASFPLAPLLALVNNILEIRVDAWKLTTQFRRLVPEKAQDIGAWQPIMQGIAILAVVTNAMIIAFTSDMIPRLVYYWSFSVPPYGDHTSYTMEGYINNTLSIFKVADFKNKSKGNPYSDLGNHTTCRYRDFRYPPGHPQEYKHNIYYWHVIAAKLAFIIVMEHVIYSVKFFISYAIPDVSKRTKSKIQREKYLTQKLLHENHLKDMTKNMGVIAERMIEAVDNNLRPKSE",
# GRAP1_HUMAN
"MAQALSEEEFQRMQAQLLELRTNNYQLSDELRKNGVELTSLRQKVAYLDKEFSKAQKALSKSKKAQEVEVLLSENEMLQAKLHSQEEDFRLQNSTLMAEFSKLCSQMEQLEQENQQLKEGAAGAGVAQAGPLVDGELLRLQAENTALQKNVAALQERYGKEAGKFSAVSEGQGDPPGGLAPTVLAPMPLAEVELKWEMEKEEKRLLWEQLQGLESSKQAETSRLQEELAKLSEKLKKKQESFCRLQTEKETLFNDSRNKIEELQQRKEADHKAQLARTQKLQQELEAANQSLAELRDQRQGERLEHAAALRALQDQVSIQSADAQEQVEGLLAENNALRTSLAALEQIQTAKTQELNMLREQTTGLAAELQQQQAEYEDLMGQKDDLNSQLQESLRANSRLLEQLQEIGQEKEQLTQELQEARKSAEKRKAMLDELAMETLQEKSQHKEELGAVRLRHEKEVLGVRARYERELRELHEDKKRQEEELRGQIREEKARTRELETLQQTVEELQAQVHSMDGAKGWFERRLKEAEESLQQQQQEQEEALKQCREQHAAELKGKEEELQDVRDQLEQAQEERDCHLKTISSLKQEVKDTVDGQRILEKKGSAALKDLKRQLHLERKRADKLQERLQDILTNSKSRSGLEELVLSEMNSPSRTQTGDSSSISSFSYREILREKESSAVPARSLSSSPQAQPPRPAELSDEEVAELFQRLAETQQEKWMLEEKVKHLEVSSASMAEDLCRKSAIIETYVMDSRIDVSVAAGHTDRSGLGSVLRDLVKPGDENLREMNKKLQNMLEEQLTKNMHLHKDMEVLSQEIVRLSKECVGPPDPDLEPGETS",
# AMOT_HUMAN
"MRNSEEQPSGGTTVLQRLLQEQLRYGNPSENRSLLAIHQQATGNGPPFPSGSGNPGPQSDVLSPQDHHQQLVAHAARQEPQGQEIQSENLIMEKQLSPRMQNNEELPTYEEAKVQSQYFRGQQHASVGAAFYVTGVTNQKMRTEGRPSVQRLNPGKMHQDEGLRDLKQGHVRSLSERLMQMSLATSGVKAHPPVTSAPLSPPQPNDLYKNPTSSSEFYKAQGPLPNQHSLKGMEHRGPPPEYPFKGMPPQSVVCKPQEPGHFYSEHRLNQPGRTEGQLMRYQHPPEYGAARPAQDISLPLSARNSQPHSPTSSLTSGGSLPLLQSPPSTRLSPARHPLVPNQGDHSAHLPRPQQHFLPNQAHQGDHYRLSQPGLSQQQQQQQQQHHHHHHHQQQQQQQPQQQPGEAYSAMPRAQPSSASYQPVPADPFAIVSRAQQMVEILSDENRNLRQELEGCYEKVARLQKVETEIQRVSEAYENLVKSSSKREALEKAMRNKLEGEIRRMHDFNRDLRERLETANKQLAEKEYEGSEDTRKTISQLFAKNKESQREKEKLEAELATARSTNEDQRRHIEIRDQALSNAQAKVVKLEEELKKKQVYVDKVEKMQQALVQLQAACEKREQLEHRLRTRLERELESLRIQQRQGNCQPTNVSEYNAAALMELLREKEERILALEADMTKWEQKYLEENVMRHFALDAAATVAAQRDTTVISHSPNTSYDTALEARIQKEEEEILMANKRCLDMEGRIKTLHAQIIEKDAMIKVLQQRSRKEPSKTEQLSCMRPAKSLMSISNAGSGLLSHSSTLTGSPIMEEKRDDKSWKGSLGILLGGDYRAEYVPSTPSPVPPSTPLLSAHSKTGSRDCSTQTERGTESNKTAAVAPISVPAPVAAAATAAAITATAATITTTMVAAAPVAVAAAAAPAAAAAPSPATAAATAAAVSPAAAGQIPAAASVASAAAVAPSAAAAAAVQVAPAAPAPVPAPALVPVPAPAAAQASAPAQTQAPTSAPAVAPTPAPTPTPAVAQAEVPASPATGPGPHRLSIPSLTCNPDKTDGPVFHSNTLERKTPIQILGQEPDAEMVEYLI",
# ACTBL_HUMAN
"MTDNELSALVVDNGSGMCKAGFGGDDAPRAVFPSMIGRPRHQGVMVGMGQKDCYVGDEAQSKRGVLTLKYPIEHGVVTNWDDMEKIWYHTFYNELRVAPDEHPILLTEAPLNPKINREKMTQIMFEAFNTPAMYVAIQAVLSLYASGRTTGIVMDSGDGVTHIVPIYEGYALPHAILRLDLAGRDLTDYLMKILTERGYNFTTTAEREIVRDVKEKLCYVALDFEQEMVRAAASSSPERSYELPDGQVITIGNERFRCPEAIFQPSFLGIESSGIHETTFNSIMKCDVDIRKDLYANTVLSGGSTMYPGIADRMQKEIITLAPSTMKIKIIAPPERKYSVWIGGSILASLSTFQQMWISKQEYDEAGPPIVHRKCF",
# IGS11_HUMAN
"MTSQRSPLAPLLLLSLHGVAASLEVSESPGSIQVARGQPAVLPCTFTTSAALINLNVIWMVTPLSNANQPEQVILYQGGQMFDGAPRFHGRVGFTGTMPATNVSIFINNTQLSDTGTYQCLVNNLPDIGGRNIGVTGLTVLVPPSAPHCQIQGSQDIGSDVILLCSSEEGIPRPTYLWEKLDNTLKLPPTATQDQVQGTVTIRNISALSSGLYQCVASNAIGTSTCLLDLQVISPQPRNIGLIAGAIGTGAVIIIFCIALILGAFFYWRSKNKEEEEEEIPNEIREDDLPPKCSSAKAFHTEISSSDNNTLTSSNAYNSRYWSNNPKVHRNTESVSHFSDLGQSFSFHSGNANIPSIYANGTHLVPGQHKTLVVTANRGSSPQVMSRSNGSVSRKPRPPHTHSYTISHATLERIGAVPVMVPAQSRAGSLV",
# IQEC2_HUMAN
"MEAGSGPPGGPGSESPNRAVEYLLELNNIIESQQQLLETQRRRIEELEGQLDQLTQENRDLREESQLHRGELHRDPHGARDSPGRESQYQNLRETQFHHRELRESQFHQAARDVGYPNREGAYQNREAVYRDKERDASYPLQDTTGYTARERDVAQCHLHHENPALGRERGGREAGPAHPGREKEAGYSAAVGVGPRPPRERGQLSRGASRSSSPGAGGGHSTSTSTSPATTLQRKSDGENSRTVSVEGDAPGSDLSTAVDSPGSQPPYRLSQLPPSSSHMGGPPAGVGLPWAQRARLQPASVALRKQEEEEIKRSKALSDSYELSTDLQDKKVEMLERKYGGSFLSRRAARTIQTAFRQYRMNKNFERLRSSASESRMSRRIILSNMRMQFSFEEYEKAQNPAYFEGKPASLDEGAMAGARSHRLERGLPYGGSCGGGIDGGGSSVTTSGEFSNDITELEDSFSKQVKSLAESIDEALNCHPSGPMSEEPGSAQLEKRESKEQQEDSSATSFSDLPLYLDDTVPQQSPERLPSTEPPPQGRPEFWAPAPLPPVPPPVPSGTREDGSREEGTRRGPGCLECRDFRLRAAHLPLLTIEPPSDSSVDLSDRSDRGSVHRQLVYEADGCSPHGTLKHKGPPGRAPIPHRHYPAPEGPAPAPPGPLPPAPNSGTGPSGVAGGRRLGKCEAAGENSDGGDNESLESSSNSNETINCSSGSSSRDSLREPPATGLCKQTYQRETRHSWDSPAFNNDVVQRRHYRIGLNLFNKKPEKGIQYLIERGFLSDTPVGVAHFILERKGLSRQMIGEFLGNRQKQFNRDVLDCVVDEMDFSSMDLDDALRKFQSHIRVQGEAQKVERLIEAFSQRYCVCNPALVRQFRNPDTIFILAFAIILLNTDMYSPSVKAERKMKLDDFIKNLRGVDNGEDIPRDLLVGIYQRIQGRELRTNDDHVSQVQAVERMIVGKKPVLSLPHRRLVCCCQLYEVPDPNRPQRLGLHQREVFLFNDLLVVTKIFQKKKILVTYSFRQSFPLVEMHMQLFQNSYYQFGIKLLSAVPGGERKVLIIFNAPSLQDRLRFTSDLRESIAEVQEMEKYRVESELEKQKGMMRPNASQPGGAKDSVNGTMARSSLEDTYGAGDGLKRGALSSSLRDLSDAGKRGRRNSVGSLDSTIEGSVISSPRPHQRMPPPPPPPPPEEYKSQRPVSNSSSFLGSLFGSKRGKGPFQMPPPPTGQASASSSSASSTHHHHHHHHHGHSHGGLGVLPDGQSKLQALHAQYCQGPGPAPPPYLPPQQPSLPPPPQQPPPLPQLGSIPPPPASAPPVGPHRHFHAHGPVPGPQHYTLGRPGRAPRRGAGGHPQFAPHGRHPLHQPTSPLPLYSPAPQHPPAHKQGPKHFIFSHHPQMMPAAGAAGGPGSRPPGGSYSHPHHPQSPLSPHSPIPPHPSYPPLPPPSPHTPHSPLPPTSPHGPLHASGPPGTANPPSANPKAKPSRISTVV",
# KCNT1_HUMAN
"MARAKLPRSPSEGKAGPGGAPAGAAAPEEPHGLSPLLPARGGGSVGSDVGQRLPVEDFSLDSSLSQVQVEFYVNENTFKERLKLFFIKNQRSSLRIRLFNFSLKLLTCLLYIVRVLLDDPALGIGCWGCPKQNYSFNDSSSEINWAPILWVERKMTLWAIQVIVAIISFLETMLLIYLSYKGNIWEQIFRVSFVLEMINTLPFIITIFWPPLRNLFIPVFLNCWLAKHALENMINDFHRAILRTQSAMFNQVLILFCTLLCLVFTGTCGIQHLERAGENLSLLTSFYFCIVTFSTVGYGDVTPKIWPSQLLVVIMICVALVVLPLQFEELVYLWMERQKSGGNYSRHRAQTEKHVVLCVSSLKIDLLMDFLNEFYAHPRLQDYYVVILCPTEMDVQVRRVLQIPLWSQRVIYLQGSALKDQDLMRAKMDNGEACFILSSRNEVDRTAADHQTILRAWAVKDFAPNCPLYVQILKPENKFHVKFADHVVCEEECKYAMLALNCICPATSTLITLLVHTSRGQEGQESPEQWQRMYGRCSGNEVYHIRMGDSKFFREYEGKSFTYAAFHAHKKYGVCLIGLKREDNKSILLNPGPRHILAASDTCFYINITKEENSAFIFKQEEKRKKRAFSGQGLHEGPARLPVHSIIASMGTVAMDLQGTEHRPTQSGGGGGGSKLALPTENGSGSRRPSIAPVLELADSSALLPCDLLSDQSEDEVTPSDDEGLSVVEYVKGYPPNSPYIGSSPTLCHLLPVKAPFCCLRLDKGCKHNSYEDAKAYGFKNKLIIVSAETAGNGLYNFIVPLRAYYRSRKELNPIVLLLDNKPDHHFLEAICCFPMVYYMEGSVDNLDSLLQCGIIYADNLVVVDKESTMSAEEDYMADAKTIVNVQTMFRLFPSLSITTELTHPSNMRFMQFRAKDSYSLALSKLEKRERENGSNLAFMFRLPFAAGRVFSISMLDTLLYQSFVKDYMITITRLLLGLDTTPGSGYLCAMKITEGDLWIRTYGRLFQKLCSSSAEIPIGIYRTESHVFSTSESQISVNVEDCEDTREVKGPWGSRAGTGGSSQGRHTGGGDPAEHPLLRRKSLQWARRLSRKAPKQAGRAAAAEWISQQRLSLYRRSERQELSELVKNRMKHLGLPTTGYEDVANLTASDVMNRVNLGYLQDEMNDHQNTLSYVLINPPPDTRLEPSDIVYLIRSDPLAHVASSSQSRKSSCSHKLSSCNPETRDETQL",
# HABP4_HUMAN
"MKGALGSPVAAAGAAMQESFGCVVANRFHQLLDDESDPFDILREAERRRQQQLQRKRRDEAAAAAGAGPRGGRSPAGASGHRAGAGGRRESQKERKSLPAPVAQRPDSPGGGLQAPGQKRTPRRGEQQGWNDSRGPEGMLERAERRSYREYRPYETERQADFTAEKFPDEKPGDRFDRDRPLRGRGGPRGGMRGRGRGGPGNRVFDAFDQRGKREFERYGGNDKIAVRTEDNMGGCGVRTWGSGKDTSDVEPTAPMEEPTVVEESQGTPEEESPAKVPELEVEEETQVQEMTLDEWKNLQEQTRPKPEFNIRKPESTVPSKAVVIHKSKYRDDMVKDDYEDDSHVFRKPANDITSQLEINFGNLPRPGRGARGGTRGGRGRIRRAENYGPRAEVVMQDVAPNPDDPEDFPALS",
# GNAS1_HUMAN
"MGVRNCLYGNNMSGQRDIPPEIGEQPEQPPLEAPGAAAPGAGPSPAEEMETEPPHNEPIPVENDGEACGPPEVSRPNFQVLNPAFREAGAHGSYSPPPEEAMPFEAEQPSLGGFWPTLEQPGFPSGVHAGLEAFGPALMEPGAFSGARPGLGGYSPPPEEAMPFEFDQPAQRGCSQLLLQVPDLAPGGPGAAGVPGAPPEEPQALRPAKAGSRGGYSPPPEETMPFELDGEGFGDDSPPPGLSRVIAQVDGSSQFAAVAASSAVRLTPAANAPPLWVPGAIGSPSQEAVRPPSNFTGSSPWMEISGPPFEIGSAPAGVDDTPVNMDSPPIALDGPPIKVSGAPDKRERAERPPVEEEAAEMEGAADAAEGGKVPSPGYGSPAAGAASADTAARAAPAAPADPDSGATPEDPDSGTAPADPDSGAFAADPDSGAAPAAPADPDSGAAPDAPADPDSGAAPDAPADPDAGAAPEAPAAPAAAETRAAHVAPAAPDAGAPTAPAASATRAAQVRRAASAAPASGARRKIHLRPPSPEIQAADPPTPRPTRASAWRGKSESSRGRRVYYDEGVASSDDDSSGDESDDGTSGCLRWFQHRRNRRRRKPQRNLLRNFLVQAFGGCFGRSESPQPKASRSLKVKKVPLAEKRRQMRKEALEKRAQKRAEKKRSKLIDKQLQDEKMGYMCTHRLLLLGAGESGKSTIVKQMRILHVNGFNGEGGEEDPQAARSNSDGEKATKVQDIKNNLKEAIETIVAAMSNLVPPVELANPENQFRVDYILSVMNVPDFDFPPEFYEHAKALWEDEGVRACYERSNEYQLIDCAQYFLDKIDVIKQADYVPSDQDLLRCRVLTSGIFETKFQVDKVNFHMFDVGGQRDERRKWIQCFNDVTAIIFVVASSSYNMVIREDNQTNRLQEALNLFKSIWNNRWLRTISVILFLNKQDLLAEKVLAGKSKIEDYFPEFARYTTPEDATPEPGEDPRVTRAKYFIRDEFLRISTASGDGRHYCYPHFTCAVDTENIRRVFNDCRDIIQRMHLRQYELL",
# PPR29_HUMAN
"MLRLGLCAAALLCVCRPGAVRADCWLIEGDKGYVWLAICSQNQPPYETIPQHINSTVHDLRLNENKLKAVLYSSLNRFGNLTDLNLTKNEISYIEDGAFLGQSSLQVLQLGYNKLSNLTEGMLRGMSRLQFLFVQHNLIEVVTPTAFSECPSLISIDLSSNRLSRLDGATFASLASLMVCELAGNPFNCECDLFGFLAWLVVFNNVTKNYDRLQCESPREFAGYPLLVPRPYHSLNAITVLQAKCRNGSLPARPVSHPTPYSTDAQREPDENSGFNPDEILSVEPPASSTTDASAGPAIKLHHVTFTSATLVVIIPHPYSKMYILVQYNNSYFSDVMTLKNKKEIVTLDKLRAHTEYTFCVTSLRNSRRFNHTCLTFTTRDPVPGDLAPSTSTTTHYIMTILGCLFGMVIVLGAVYYCLRKRRMQEEKQKSVNVKKTILEMRYGADVDAGSIVHAAQKLGEPPVLPVSRMASIPSMIGEKLPTAKGLEAGLDTPKVATKGNYIEVRTGAGGDGLARPEDDLPDLENGQGSAAEISTIAKEVDKVNQIINNCIDALKLDSASFLGGGSSSGDPELAFECQSLPAAAAASSATGPGALERPSFLSPPYKESSHHPLQRQLSADAAVTRKTCSVSSSGSIKSAKVFSLDVPDHPAATGLAKGDSKYIEKGSPLNSPLDRLPLVPAGSGGGSGGGGGIHHLEVKPAYHCSEHRHSFPALYYEEGADSLSQRVSFLKPLTRSKRDSTYSQLSPRHYYSGYSSSPEYSSESTHKIWERFRPYKKHHREEVYMAAGHALRKKVQFAKDEDLHDILDYWKGVSAQQKL",
# LRRK2_HUMAN
"MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFTYSERASKLFQGKNIHVPLLIVLDSYMRVASVQQVGWSLLCKLIEVCPGTMQSLMGPQDVGNDWEVLGVHQLILKMLTVHNASVNLSVIGLKTLDLLLTSGKITLLILDEESDIFMLIFDAMHSFPANDEVQKLGCKALHVLFERVSEEQLTEFVENKDYMILLSALTNFKDEEEIVLHVLHCLHSLAIPCNNVEVLMSGNVRCYNIVVEAMKAFPMSERIQEVSCCLLHRLTLGNFFNILVLNEVHEFVVKAVQQYPENAALQISALSCLALLTETIFLNQDLEEKNENQENDDEGEEDKLFWLEACYKALTWHRKNKHVQEAACWALNNLLMYQNSLHEKIGDEDGHFPAHREVMLSMLMHSSSKEVFQASANALSTLLEQNVNFRKILLSKGIHLNVLELMQKHIHSPEVAESGCKMLNHLFEGSNTSLDIMAAVVPKILTVMKRHETSLPVQLEALRAILHFIVPGMPEESREDTEFHHKLNMVKKQCFKNDIHKLVLAALNRFIGNPGIQKCGLKVISSIVHFPDALEMLSLEGAMDSVLHTLQMYPDDQEIQCLGLSLIGYLITKKNVFIGTGHLLAKILVSSLYRFKDVAEIQTKGFQTILAILKLSASFSKLLVHHSFDLVIFHQMSSNIMEQKDQQFLNLCCKCFAKVAMDDYLKNVMLERACDQNNSIMVECLLLLGADANQAKEGSSLICQVCEKESSPKLVELLLNSGSREQDVRKALTISIGKGDSQIISLLLRRLALDVANNSICLGGFCIGKVEPSWLGPLFPDKTSNLRKQTNIASTLARMVIRYQMKSAVEEGTASGSDGNFSEDVLSKFDEWTFIPDSSMDSVFAQSDDLDSEGSEGSFLVKKKSNSISVGEFYRDAVLQRCSPNLQRHSNSLGPIFDHEDLLKRKRKILSSDDSLRSSKLQSHMRHSDSISSLASEREYITSLDLSANELRDIDALSQKCCISVHLEHLEKLELHQNALTSFPQQLCETLKSLTHLDLHSNKFTSFPSYLLKMSCIANLDVSRNDIGPSVVLDPTVKCPTLKQFNLSYNQLSFVPENLTDVVEKLEQLILEGNKISGICSPLRLKELKILNLSKNHISSLSENFLEACPKVESFSARMNFLAAMPFLPPSMTILKLSQNKFSCIPEAILNLPHLRSLDMSSNDIQYLPGPAHWKSLNLRELLFSHNQISILDLSEKAYLWSRVEKLHLSHNKLKEIPPEIGCLENLTSLDVSYNLELRSFPNEMGKLSKIWDLPLDELHLNFDFKHIGCKAKDIIRFLQQRLKKAVPYNRMKLMIVGNTGSGKTTLLQQLMKTKKSDLGMQSATVGIDVKDWPIQIRDKRKRDLVLNVWDFAGREEFYSTHPHFMTQRALYLAVYDLSKGQAEVDAMKPWLFNIKARASSSPVILVGTHLDVSDEKQRKACMSKITKELLNKRGFPAIRDYHFVNATEESDALAKLRKTIINESLNFKIRDQLVVGQLIPDCYVELEKIILSERKNVPIEFPVIDRKRLLQLVRENQLQLDENELPHAVHFLNESGVLLHFQDPALQLSDLYFVEPKWLCKIMAQILTVKVEGCPKHPKGIISRRDVEKFLSKKRKFPKNYMSQYFKLLEKFQIALPIGEEYLLVPSSLSDHRPVIELPHCENSEIIIRLYEMPYFPMGFWSRLINRLLEISPYMLSGRERALRPNRMYWRQGIYLNWSPEAYCLVGSEVLDNHPESFLKITVPSCRKGCILLGQVVDHIDSLMEEWFPGLLEIDICGEGETLLKKWALYSFNDGEEHQKILLDDLMKKAEEGDLLVNPDQPRLTIPISQIAPDLILADLPRNIMLNNDELEFEQAPEFLLGDGSFGSVYRAAYEGEEVAVKIFNKHTSLRLLRQELVVLCHLHHPSLISLLAAGIRPRMLVMELASKGSLDRLLQQDKASLTRTLQHRIALHVADGLRYLHSAMIIYRDLKPHNVLLFTLYPNAAIIAKIADYGIAQYCCRMGIKTSEGTPGFRAPEVARGNVIYNQQADVYSFGLLLYDILTTGGRIVEGLKFPNEFDELEIQGKLPDPVKEYGCAPWPMVEKLIKQCLKENPQERPTSAQVFDILNSAELVCLTRRILLPKNVIVECMVATHHNSRNASIWLGCGHTDRGQLSFLDLNTEGYTSEEVADSRILCLALVHLPVEKESWIVSGTQSGTLLVINTEDGKKRHTLEKMTDSVTCLYCNSFSKQSKQKNFLLVGTADGKLAIFEDKTVKLKGAAPLKILNIGNVSTPLMCLSESTNSTERNVMWGGCGTKIFSFSNDFTIQKLIETRTSQLFSYAAFSDSNIITVVVDTALYIAKQNSPVVEVWDKKTEKLCGLIDCVHFLREVMVKENKESKHKMSYSGRVKTLCLQKNTALWIGTGGGHILLLDLSTRRLIRVIYNFCNSVRVMMTAQLGSLKNVMLVLGYNRKNTEGTQKQKEIQSCLTVWDINLPHEVQNLEKHIEVRKELAEKMRRTSVE",
# SNP47_HUMAN
"MRAARRGLHCAGAERPRRRGRLWDSSGVPQRQKRPGPWRTQTQEQMSRDVCIHTWPCTYYLEPKRRWVTGQLSLTSLSLRFMTDSTGEILVSFPLSSIVEIKKEASHFIFSSITILEKGHAKHWFSSLRPSRNVVFSIIEHFWRELLLSQPGAVADASVPRTRGEELTGLMAGSQKRLEDTARVLHHQGQQLDSVMRGLDKMESDLEVADRLLTELESPAWWPFSSKLWKTPPETKPREDVSMTSCEPFGKEGILIKIPAVISHRTESHVKPGRLTVLVSGLEIHDSSSLLMHRFEREDVDDIKVHSPYEISIRQRFIGKPDMAYRLISAKMPEVIPILEVQFSKKMELLEDALVLRSARTSSPAEKSCSVWHAASGLMGRTLHREPPAGDQEGTALHLQTSLPALSEADTQELTQILRRMKGLALEAESELERQDEALDGVAAAVDRATLTIDKHNRRMKRLT",
# TM240_HUMAN
"MSMSANTMIFMILGASVVMAIACLMDMNALLDRFHNYILPHLRGEDRVCHCNCGRHHIHYVIPYDGDQSVVDASENYFVTDSVTKQEIDLMLGLLLGFCISWFLVWMDGVLHCAVRAWRAGRRYDGSWTWLPKLCSLRELGRRPHRPFEEAAGNMVHVKQKLYHNGHPSPRHL",
# TPRGL_HUMAN
"MLQLRDSVDSAGTSPTAVLAAGEEVGAGGGPGGGRPGAGTPLRQTLWPLSIHDPTRRARVKEYFVFRPGSIEQAVEEIRVVVRPVEDGEIQGVWLLTEVDHWNNEKERLVLVTEQSLLICKYDFISLQCQQVVRIALNAVDTISYGEFQFPPKSLNKREGFGIRIQWDKQSRPSFINRWNPWSTNVPYATFTEHPMAGADEKTASLCQLESFKALLIQAVKKAQKESPLPGQANGVLILERPLLIETYVGLMSFINNEAKLGYSMTRGKIGF",
# STXB5_HUMAN
"MRKFNIRKVLDGLTAGSSSASQQQQQQHPPGNREPEIQETLQSEHFQLCKTVRHGFPYQPSALAFDPVQKILAVGTQTGALRLFGRPGVECYCQHDSGAAVIQLQFLINEGALVSALADDTLHLWNLRQKRPAILHSLKFCRERVTFCHLPFQSKWLYVGTERGNIHIVNVESFTLSGYVIMWNKAIELSSKSHPGPVVHISDNPMDEGKLLIGFESGTVVLWDLKSKKADYRYTYDEAIHSVAWHHEGKQFICSHSDGTLTIWNVRSPAKPVQTITPHGKQLKDGKKPEPCKPILKVEFKTTRSGEPFIILSGGLSYDTVGRRPCLTVMHGKSTAVLEMDYSIVDFLTLCETPYPNDFQEPYAVVVLLEKDLVLIDLAQNGYPIFENPYPLSIHESPVTCCEYFADCPVDLIPALYSVGARQKRQGYSKKEWPINGGNWGLGAQSYPEIIITGHADGSVKFWDASAITLQVLYKLKTSKVFEKSRNKDDRPNTDIVDEDPYAIQIISWCPESRMLCIAGVSAHVIIYRFSKQEVITEVIPMLEVRLLYEINDVETPEGEQPPPLPTPVGGSNPQPIPPQSHPSTSSSSSDGLRDNVPCLKVKNSPLKQSPGYQTELVIQLVWVGGEPPQQITSLAVNSSYGLVVFGNCNGIAMVDYLQKAVLLNLGTIELYGSNDPYRREPRSPRKSRQPSGAGLCDISEGTVVPEDRCKSPTSGSSSPHNSDDEQKMNNFIEKVKTKSRKFSKMVANDIAKMSRKLSLPTDLKPDLDVKDNSFSRSRSSSVTSIDKESREAISALHFCETFTRKTDSSPSPCLWVGTTLGTVLVIALNLPPGGEQRLLQPVIVSPSGTILRLKGAILRMAFLDTTGCLIPPAYEPWREHNVPEEKDEKEKLKKRRPVSVSPSSSQEISENQYAVICSEKQAKVISLPTQNCAYKQNITETSFVLRGDIVALSNSICLACFCANGHIMTFSLPSLRPLLDVYYLPLTNMRIARTFCFTNNGQALYLVSPTEIQRLTYSQETCENLQEMLGELFTPVETPEAPNRGFFKGLFGGGAQSLDREELFGESSSGKASRSLAQHIPGPGGIEGVKGAASGVVGELARARLALDERGQKLGDLEERTAAMLSSAESFSKHAHEIMLKYKDKKWYQF",
# SYT6_HUMAN
"MSGVWGAGGPRCQEALAVLASLCRARPPPLGLDVETCRSFELQPPERSPSAAGAGTSVSLLAVVVIVCGVALVAVFLFLFWKLCWMPWRNKEASSPSSANPPLEALQSPSFRGNMADKLKDPSTLGFLEAAVKISHTSPDIPAEVQMSVKEHIMRHTRLQRQTTEPASSTRHTSFKRHLPRQMHVSSVDYGNELPPAAEQPTSIGRIKPELYKQKSVDGEDAKSEATKSCGKINFSLRYDYETETLIVRILKAFDLPAKDFCGSSDPYVKIYLLPDRKCKLQTRVHRKTLNPTFDENFHFPVPYEELADRKLHLSVFDFDRFSRHDMIGEVILDNLFEASDLSRETSIWKDIQYATSESVDLGEIMFSLCYLPTAGRLTLTVIKCRNLKAMDITGYSDPYVKVSLLCDGRRLKKKKTTIKKNTLNPVYNEAIIFDIPPENMDQVSLLISVMDYDRVGHNEIIGVCRVGITAEGLGRDHWNEMLAYPRKPIAHWHSLVEVKKSFKEGNPRL",
# MGLYR_HUMAN
"MGAMAYPLLLCLLLAQLGLGAVGASRDPQGRPDSPRERTPKGKPHAQQPGRASASDSSAPWSRSTDGTILAQKLAEEVPMDVASYLYTGDSHQLKRANCSGRYELAGLPGKWPALASAHPSLHRALDTLTHATNFLNVMLQSNKSREQNLQDDLDWYQALVWSLLEGEPSISRAAITFSTDSLSAPAPQVFLQATREESRILLQDLSSSAPHLANATLETEWFHGLRRKWRPHLHRRGPNQGPRGLGHSWRRKDGLGGDKSHFKWSPPYLECENGSYKPGWLVTLSSAIYGLQPNLVPEFRGVMKVDINLQKVDIDQCSSDGWFSGTHKCHLNNSECMPIKGLGFVLGAYECICKAGFYHPGVLPVNNFRRRGPDQHISGSTKDVSEEAYVCLPCREGCPFCADDSPCFVQEDKYLRLAIISFQALCMLLDFVSMLVVYHFRKAKSIRASGLILLETILFGSLLLYFPVVILYFEPSTFRCILLRWARLLGFATVYGTVTLKLHRVLKVFLSRTAQRIPYMTGGRVMRMLAVILLVVFWFLIGWTSSVCQNLEKQISLIGQGKTSDHLIFNMCLIDRWDYMTAVAEFLFLLWGVYLCYAVRTVPSAFHEPRYMAVAVHNELIISAIFHTIRFVLASRLQSDWMLMLYFAHTHLTVTVTIGLLLIPKFSHSSNNPRDDIATEAYEDELDMGRSGSYLNSSINSAWSEHSLDPEDIRDELKKLYAQLEIYKRKKMITNNPHLQKKRCSKKGLGRSIMRRITEIPETVSRQCSKEDKEGADHGTAKGTALIRKNPPESSGNTGKSKEETLKNRVFSLKKSHSTYDHVRDQTEESSSLPTESQEEETTENSTLESLSGKKLTQKLKEDSEAESTESVPLVCKSASAHNLSSEKKTGHPRTSMLQKSLSVIASAKEKTLGLAGKTQTAGVEERTKSQKPLPKDKETNRNHSNSDNTETKDPAPQNSNPAEEPRKPQKSGIMKQQRVNPTTANSDLNPGTTQMKDNFDIGEVCPWEVYDLTPGPVPSESKVQKHVSIVASEMEKNPTFSLKEKSHHKPKAAEVCQQSNQKRIDKAEVCLWESQGQSILEDEKLLISKTPVLPERAKEENGGQPRAANVCAGQSEELPPKAVASKTENENLNQIGHQEKKTSSSEENVRGSYNSSNNFQQPLTSRAEVCPWEFETPAQPNAGRSVALPASSALSANKIAGPRKEEIWDSFKV",
# WLS_HUMAN
"MAGAIIENMSTKKLCIVGGILLVFQIIAFLVGGLIAPGPTTAVSYMSVKCVDARKNHHKTKWFVPWGPNHCDKIRDIEEAIPREIEANDIVFSVHIPLPHMEMSPWFQFMLFILQLDIAFKLNNQIRENAEVSMDVSLAYRDDAFAEWTEMAHERVPRKLKCTFTSPKTPEHEGRYYECDVLPFMEIGSVAHKFYLLNIRLPVNEKKKINVGIGEIKDIRLVGIHQNGGFTKVWFAMKTFLTPSIFIIMVWYWRRITMMSRPPVLLEKVIFALGISMTFINIPVEWFSIGFDWTWMLLFGDIRQGIFYAMLLSFWIIFCGEHMMDQHERNHIAGYWKQVGPIAVGSFCLFIFDMCERGVQLTNPFYSIWTTDIGTELAMAFIIVAGICLCLYFLFLCFMVFQVFRNISGKQSSLPAMSKVRRLHYEGLIFRFKFLMLITLACAAMTVIFFIVSQVTEGHWKWGGVTVQVNSAFFTGIYGMWNLYVFALMFLYAPSHKNYGEDQSNGDLGVHSGEELQLTTTITHVDGPTEIYKLTRKEAQE",
# MAGI3_HUMAN
"MSKTLKKKKHWLSKVQECAVSWAGPPGDFGAEIRGGAERGEFPYLGRLREEPGGGTCCVVSGKAPSPGDVLLEVNGTPVSGLTNRDTLAVIRHFREPIRLKTVKPGKVINKDLRHYLSLQFQKGSIDHKLQQVIRDNLYLRTIPCTTRAPRDGEVPGVDYNFISVEQFKALEESGALLESGTYDGNFYGTPKPPAEPSPFQPDPVDQVLFDNEFDAESQRKRTTSVSKMERMDSSLPEEEEDEDKEAINGSGNAENRERHSESSDWMKTVPSYNQTNSSMDFRNYMMRDETLEPLPKNWEMAYTDTGMIYFIDHNTKTTTWLDPRLCKKAKAPEDCEDGELPYGWEKIEDPQYGTYYVDHLNQKTQFENPVEEAKRKKQLGQVEIGSSKPDMEKSHFTRDPSQLKGVLVRASLKKSTMGFGFTIIGGDRPDEFLQVKNVLKDGPAAQDGKIAPGDVIVDINGNCVLGHTHADVVQMFQLVPVNQYVNLTLCRGYPLPDDSEDPVVDIVAATPVINGQSLTKGETCMNPQDFKPGAMVLEQNGKSGHTLTGDGLNGPSDASEQRVSMASSGSSQPELVTIPLIKGPKGFGFAIADSPTGQKVKMILDSQWCQGLQKGDIIKEIYHQNVQNLTHLQVVEVLKQFPVGADVPLLILRGGPPSPTKTAKMKTDKKENAGSLEAINEPIPQPMPFPPSIIRSGSPKLDPSEVYLKSKTLYEDKPPNTKDLDVFLRKQESGFGFRVLGGDGPDQSIYIGAIIPLGAAEKDGRLRAADELMCIDGIPVKGKSHKQVLDLMTTAARNGHVLLTVRRKIFYGEKQPEDDSSQAFISTQNGSPRLNRAEVPARPAPQEPYDVVLQRKENEGFGFVILTSKNKPPPGVIPHKIGRVIEGSPADRCGKLKVGDHISAVNGQSIVELSHDNIVQLIKDAGVTVTLTVIAEEEHHGPPSGTNSARQSPALQHRPMGQSQANHIPGDRSALEGEIGKDVSTSYRHSWSDHKHLAQPDTAVISVVGSRHNQNLGCYPVELERGPRGFGFSLRGGKEYNMGLFILRLAEDGPAIKDGRIHVGDQIVEINGEPTQGITHTRAIELIQAGGNKVLLLLRPGTGLIPDHGDWDINNPSSSNVIYDEQSPLPPSSHFASIFEESHVPVIEESLRVQICEKAEELKDIVPEKKSTLNENQPEIKHQSLLQKNVSKRDPPSSHGHSNKKNLLKVENGVTRRGRSVSPKKPASQHSEEHLDKIPSPLKNNPKRRPRDQSLSPSKGENKSCQVSTRAGSGQDQCRKSRGRSASPKKQQKIEGSKAPSNAEAKLLEGKSRRIAGYTGSNAEQIPDGKEKSDVIRKDAKQNQLEKSRTRSPEKKIKRMVEKSLPSKMTNKTTSKEVSENEKGKKVTTGETSSSNDKIGENVQLSEKRLKQEPEEKVVSNKTEDHKGKELEAADKNKETGRFKPESSSPVKKTLITPGPWKVPSGNKVTGTIGMAEKRQ",
# AB17B_HUMAN
"MNNLSFSELCCLFCCPPCPGKIASKLAFLPPDPTYTLMCDESGSRWTLHLSERADWQYSSREKDAIECFMTRTSKGNRIACMFVRCSPNAKYTLLFSHGNAVDLGQMSSFYIGLGSRINCNIFSYDYSGYGASSGKPTEKNLYADIEAAWLALRTRYGIRPENVIIYGQSIGTVPSVDLAARYESAAVILHSPLTSGMRVAFPDTKKTYCFDAFPNIDKISKITSPVLIIHGTEDEVIDFSHGLALFERCQRPVEPLWVEGAGHNDVELYGQYLERLKQFVSQELVNL",
# SYDE2_HUMAN
"MHDLPPDSGARRGGRGLADHSFPAGARAPGQPPSRGAAYRRACPRDGERGGGGRPRQQVSPPRSPQREPRGGQLRTPRMRPSCSRSLESLRVGAKPPPFQRWPSDSWIRCGAHRDWDEPPPRGGRMDGWSGDRARAAAPTGLQPPGCKDHGCSSGSPFRDPAGSSVIRSGKGDRQEGPSFLRPPAVTVKKLQKWMYKGRLLSLGMKGRARGTAPKVTGTQAASPNVGALKVRENRVLSVPPDQRITLTDLFENAYGSSMKGRELEELKDNIEFRGHKPLNSITVSKKRNWLYQSTLRPLNLEEENKKCQDRSHLSISPVSLPKHQLSQSFLKSSKEYCTYVVCNATNSSLSKNCALDFNEENDADDEGEIWYNPIPEDDDLGISSALSFGEADSAVLKLPAVNLSMLSGSDLMKAERHTEDSLCSSEHAGDIQTTRSNGMNPIHPAHSTEFVQQYKQKLGHKTQEGIMVEDSPMLKSPFAGSGILAATNSTELGIMEPSSPNPSPVKKGSSINWSLPDKIKSPRTVRKLSMKMKKLPEFSRKLSVKGTLNYINSPDNTPSLSKYNCREVHHTDILPSGNTTTAAKRNVISRYHLDTSVSSQQSYQKKNSMSSKYSCKGGYLSDGDSPELTTKASKHGSENKFGKGKEIISNSCSKNEIDIDAFRHYSFSDQPKCSQYISGLMSVHFYGAEDLKPPRIDSKDVFCAIQVDSVNKARTALLTCRTTFLDMDHTFNIEIENAQHLKLVVFSWEPTPRKNRVCCHGTVVLPTLFRVTKTHQLAVKLEPRGLIYVKVTLMEQWENSLHGLDINQEPIIFGVDIQKVVEKENIGLMVPLLIQKCIMEIEKRGCQVVGLYRLCGSAAVKKELREAFERDSKAVGLCENQYPDINVITGVLKDYLRELPSPLITKQLYEAVLDAMAKSPLKMSSNGCENDPGDSKYTVDLLDCLPEIEKATLKMLLDHLKLVASYHEVNKMTCQNLAVCFGPVLLSQRQEPSTHNNRVFTDSEELASALDFKKHIEVLHYLLQLWPVQRLTVKKSTDNLFPEQKSSLNYLRQKKERPHMLNLSGTDSSGVLRPRQNRLDSPLSNRYAGDWSSCGENYFLNTKENLNDVDYDDVPSEDRKIGENYSKMDGPEVMIEQPIPMSKECTFQTYLTMQTVESTVDRKNNLKDLQESIDTLIGNLERELNKNKLNMSF",
# RN220_HUMAN
"MDLHRAAFKMENSSYLPNPLASPALMVLASTAEASRDASIPCQQPRPFGVPVSVDKDVHIPFTNGSYTFASMYHRQGGVPGTFANRDFPPSLLHLHPQFAPPNLDCTPISMLNHSGVGAFRPFASTEDRESYQSAFTPAKRLKNCHDTESPHLRFSDADGKEYDFGTQLPSSSPGSLKVDDTGKKIFAVSGLISDREASSSPEDRNDRCKKKAAALFDSQAPICPICQVLLRPSELQEHMEQELEQLAQLPSSKNSLLKDAMAPGTPKSLLLSASIKREGESPTASPHSSATDDLHHSDRYQTFLRVRANRQTRLNARIGKMKRRKQDEGQREGSCMAEDDAVDIEHENNNRFEEYEWCGQKRIRATTLLEGGFRGSGFIMCSGKENPDSDADLDVDGDDTLEYGKPQYTEADVIPCTGEEPGEAKEREALRGAVLNGGPPSTRITPEFSKWASDEMPSTSNGESSKQEAMQKTCKNSDIEKITEDSAVTTFEALKARVRELERQLSRGDRYKCLICMDSYSMPLTSIQCWHVHCEECWLRTLGAKKLCPQCNTITAPGDLRRIYL",
# C1QL3_HUMAN
"MVLLLVILIPVLVSSAGTSAHYEMLGTCRMVCDPYGGTKAPSTAATPDRGLMQSLPTFIQGPKGEAGRPGKAGPRGPPGEPGPPGPMGPPGEKGEPGRQGLPGPPGAPGLNAAGAISAATYSTVPKIAFYAGLKRQHEGYEVLKFDDVVTNLGNHYDPTTGKFTCSIPGIYFFTYHVLMRGGDGTSMWADLCKNNQVRASAIAQDADQNYDYASNSVVLHLEPGDEVYIKLDGGKAHGGNNNKYSTFSGFIIYAD",
# SUSD4_HUMAN
"MYHGMNPSNGDGFLEQQQQQQQPQSPQRLLAVILWFQLALCFGPAQLTGGFDDLQVCADPGIPENGFRTPSGGVFFEGSVARFHCQDGFKLKGATKRLCLKHFNGTLGWIPSDNSICVQEDCRIPQIEDAEIHNKTYRHGEKLIITCHEGFKIRYPDLHNMVSLCRDDGTWNNLPICQGCLRPLASSNGYVNISELQTSFPVGTVISYRCFPGFKLDGSAYLECLQNLIWSSSPPRCLALEVCPLPPMVSHGDFVCHPRPCERYNHGTVVEFYCDPGYSLTSDYKYITCQYGEWFPSYQVYCIKSEQTWPSTHETLLTTWKIVAFTATSVLLVLLLVILARMFQTKFKAHFPPRGPPRSSSSDPDFVVVDGVPVMLPSYDEAVSGGLSALGPGYMASVGQGCPLPVDDQSPPAYPGSGDTDTGPGESETCDSVSGSSELLQSLYSPPRCQESTHPASDNPDIIASTAEEVASTSPGIDIADEIPLMEEDP",
# BTBD8_HUMAN
"MARCGEGSAAPMVLLGSAGVCSKGLQRKGPCERRRLKATVSEQLSQDLLRLLREEFHTDVTFSVGCTLFKAHKAVLLARVPDFYFHTIGQTSNSLTNQEPIAVENVEALEFRTFLQIIYSSNRNIKNYEEEILRKKIMEIGISQKQLDISFPKCENSSDCSLQKHEIPEDISDRDDDFISNDNYDLEPASELGEDLLKLYVKPCCPDIDIFVDGKRFKAHRAILSARSSYFAAMLSGCWAESSQEYVTLQGISHVELNVMMHFIYGGTLDIPDKTNVGQILNMADMYGLEGLKEVAIYILRRDYCNFFQKPVPRTLTSILECLIIAHSVGVESLFADCMKWIVKHFARFWSERSFANIPPEIQKSCLNMLIQSLNDKNAAFLLMESDRLIISLPRVKWTEAALTMASQLQEKCIAFIVDNFSKIIQSENFALLLQSQAMSSTADLLDTILKAIEENITTENSCSLLMALDTLLNSDSTKEMGFTCKIQALRDKLWIFLVQSFYAVRHTESWKLMSTDDQQKIQAAAFDKGDDRRLGKKPIFSSSQQRKQVSDSGDIKIKSWRGNNKKECWSYLSTNKKMKSDGLGASGHSSSTNRNSINKTLKQDDVKEKDGTKIASKITKELKTGGKNVSGKPKTVTKSKTENGDKARLENMSPRQVVERSATAAAAATGQKNLLNGKGVRNQEGQISGARPKVLTGNLNVQAKAKPLKKATGKDSPCLSIAGPSSRSTDSSMEFSISTECLDEPKENGSTEEEKPSGHKLSFCDSPGQMMKNSVDSVKNSTVAIKSRPVSRVTNGTSNKKSIHEQDTNVNNSVLKKVSGKGCSEPVPQAILKKRGTSNGCTAAQQRTKSTPSNLTKTQGSQGESPNSVKSSVSSRQSDENVAKLDHNTTTEKQAPKRKMVKQVHTALPKVNAKIVAMPKNLNQSKKGETLNNKDSKQKMPPGQVISKTQPSSQRPLKHETSTVQKSMFHDVRDNNNKDSVSEQKPHKPLINLASEISDAEALQSSCRPDPQKPLNDQEKEKLALECQNISKLDKSLKHELESKQICLDKSETKFPNHKETDDCDAANICCHSVGSDNVNSKFYSTTALKYMVSNPNENSLNSNPVCDLDSTSAGQIHLISDRENQVGRKDTNKQSSIKCVEDVSLCNPERTNGTLNSAQEDKKSKVPVEGLTIPSKLSDESAMDEDKHATADSDVSSKCFSGQLSEKNSPKNMETSESPESHETPETPFVGHWNLSTGVLHQRESPESDTGSATTSSDDIKPRSEDYDAGGSQDDDGSNDRGISKCGTMLCHDFLGRSSSDTSTPEELKIYDSNLRIEVKMKKQSNNDLFQVNSTSDDEIPRKRPEIWSRSAIVHSRERENIPRGSVQFAQEIDQVSSSADETEDERSEAENVAENFSISNPAPQQFQGIINLAFEDATENECREFSATKKFKRSVLLSVDECEELGSDEGEVHTPFQASVDSFSPSDVFDGISHEHHGRTCYSRFSRESEDNILECKQNKGNSVCKNESTVLDLSSIDSSRKNKQSVSATEKKNTIDVLSSRSRQLLREDKKVNNGSNVENDIQQRSKFLDSDVKSQERPCHLDLHQREPNSDIPKNSSTKSLDSFRSQVLPQEGPVKESHSTTTEKANIALSAGDIDDCDTLAQTRMYDHRPSKTLSPIYEMDVIEAFEQKVESETHVTDMDFEDDQHFAKQDWTLLKQLLSEQDSNLDVTNSVPEDLSLAQYLINQTLLLARDSSKPQGITHIDTLNRWSELTSPLDSSASITMASFSSEDCSPQGEWTILELETQH",
# ANO1_HUMAN
"MRVNEKYSTLPAEDRSVHIINICAIEDIGYLPSEGTLLNSLSVDPDAECKYGLYFRDGRRKVDYILVYHHKRPSGNRTLVRRVQHSDTPSGARSVKQDHPLPGKGASLDAGSGEPPMDYHEDDKRFRREEYEGNLLEAGLELERDEDTKIHGVGFVKIHAPWNVLCREAEFLKLKMPTKKMYHINETRGLLKKINSVLQKITDPIQPKVAEHRPQTMKRLSYPFSREKQHLFDLSDKDSFFDSKTRSTIVYEILKRTTCTKAKYSMGITSLLANGVYAAAYPLHDGDYNGENVEFNDRKLLYEEWARYGVFYKYQPIDLVRKYFGEKIGLYFAWLGVYTQMLIPASIVGIIVFLYGCATMDENIPSMEMCDQRHNITMCPLCDKTCSYWKMSSACATARASHLFDNPATVFFSVFMALWAATFMEHWKRKQMRLNYRWDLTGFEEEEEAVKDHPRAEYEARVLEKSLKKESRNKEKRRHIPEESTNKWKQRVKTAMAGVKLTDKVKLTWRDRFPAYLTNLVSIIFMIAVTFAIVLGVIIYRISMAAALAMNSSPSVRSNIRVTVTATAVIINLVVIILLDEVYGCIARWLTKIEVPKTEKSFEERLIFKAFLLKFVNSYTPIFYVAFFKGRFVGRPGDYVYIFRSFRMEECAPGGCLMELCIQLSIIMLGKQLIQNNLFEIGIPKMKKLIRYLKLKQQSPPDHEECVKRKQRYEVDYNLEPFAGLTPEYMEMIIQFGFVTLFVASFPLAPLFALLNNIIEIRLDAKKFVTELRRPVAVRAKDIGIWYNILRGIGKLAVIINAFVISFTSDFIPRLVYLYMYSKNGTMHGFVNHTLSSFNVSDFQNGTAPNDPLDLGYEVQICRYKDYREPPWSENKYDISKDFWAVLAARLAFVIVFQNLVMFMSDFVDWVIPDIPKDISQQIHKEKVLMVELFMREEQDKQQLLETWMEKERQKDEPPCNHHNTKACPDSLGSPAPSHAYHGGVL",
# PLCX3_HUMAN
"MASSQGKNELKLADWMATLPESMHSIPLTNLAIPGSHDSFSFYIDEASPVGPEQPETVQNFVSVFGTVAKKLMRKWLATQTMNFTGQLGAGIRYFDLRISTKPRDPDNELYFAHGLFSAKVNEGLEEINAFLTDHHKEVVFLDFNHFYGMQKYHHEKLVQMLKDIYGNKMCPAIFAQEVSLKYLWEKDYQVLVFYHSPVALEVPFLWPGQMMPAPWANTTDPEKLIQFLQASITERRKKGSFFISQVVLTPKASTVVKGVASGLRETITERALPAMMQWVRTQKPGESGINIVTADFVELGDFISTVIKLNYVFDEGEANT",
# A16L1_HUMAN
"MSSGLRAADFPRWKRHISEQLRRRDRLQRQAFEEIILQYNKLLEKSDLHSVLAQKLQAEKHDVPNRHEISPGHDGTWNDNQLQEMAQLRIKHQEELTELHKKRGELAQLVIDLNNQMQRKDREMQMNEAKIAECLQTISDLETECLDLRTKLCDLERANQTLKDEYDALQITFTALEGKLRKTTEENQELVTRWMAEKAQEANRLNAENEKDSRRRQARLQKELAEAAKEPLPVEQDDDIEVIVDETSDHTEETSPVRAISRAATKRLSQPAGGLLDSITNIFGRRSVSSFPVPQDNVDTHPGSGKEVRVPATALCVFDAHDGEVNAVQFSPGSRLLATGGMDRRVKLWEVFGEKCEFKGSLSGSNAGITSIEFDSAGSYLLAASNDFASRIWTVDDYRLRHTLTGHSGKVLSAKFLLDNARIVSGSHDRTLKLWDLRSKVCIKTVFAGSSCNDIVCTEQCVMSGHFDKKIRFWDIRSESIVREMELLGKITALDLNPERTELLSCSRDDLLKVIDLRTNAIKQTFSAPGFKCGSDWTRVVFSPDGSYVAAGSAEGSLYIWSVLTGKVEKVLSKQHSSSINAVAWSPSGSHVVSVDKGCKAVLWAQY",
# KCD16_HUMAN
"MALSGNCSRYYPREQGSAVPNSFPEVVELNVGGQVYFTRHSTLISIPHSLLWKMFSPKRDTANDLAKDSKGRFFIDRDGFLFRYILDYLRDRQVVLPDHFPEKGRLKREAEYFQLPDLVKLLTPDEIKQSPDEFCHSDFEDASQGSDTRICPPSSLLPADRKWGFITVGYRGSCTLGREGQADAKFRRVPRILVCGRISLAKEVFGETLNESRDPDRAPERYTSRFYLKFKHLERAFDMLSECGFHMVACNSSVTASFINQYTDDKIWSSYTEYVFYREPSRWSPSHCDCCCKNGKGDKEGESGTSCNDLSTSSCDSQSEASSPQETVICGPVTRQTNIQTLDRPIKKGPVQLIQQSEMRRKSDLLRTLTSGSRESNMSSKKKAVKEKLSIEEELEKCIQDFLKIKIPDRFPERKHPWQSELLRKYHL",
# MCTP2_HUMAN
"MDLDKPSVWGSLKQRTRPLLINLSKKKVKKNPSKPPDLRARHHLDRRLSLSVPDLLEAEALAPEGRPYSGPQSSYTSVPSSLSTAGIFPKSSSSSLKQSEEELDWSQEEASHLHVVETDSEEAYASPAERRRVSSNGIFDLQKTSLGGDAPEEPEKLCGSSDLNASMTSQHFEEQSVPGEASDGLSNLPSPFAYLLTIHLKEGRNLVVRDRCGTSDPYVKFKLNGKTLYKSKVIYKNLNPVWDEIVVLPIQSLDQKLRVKVYDRDLTTSDFMGSAFVILSDLELNRTTEHILKLEDPNSLEDDMGVIVLNLNLVVKQGDFKRHRWSNRKRLSASKSSLIRNLRLSESLKKNQLWNGIISITLLEGKNVSGGSMTEMFVQLKLGDQRYKSKTLCKSANPQWQEQFDFHYFSDRMGILDIEVWGKDNKKHEERLGTCKVDISALPLKQANCLELPLDSCLGALLMLVTLTPCAGVSVSDLCVCPLADLSERKQITQRYCLQNSLKDVKDVGILQVKVLKAADLLAADFSGKSDPFCLLELGNDRLQTHTVYKNLNPEWNKVFTFPIKDIHDVLEVTVFDEDGDKPPDFLGKVAIPLLSIRDGQPNCYVLKNKDLEQAFKGVIYLEMDLIYNPVKASIRTFTPREKRFVEDSRKLSKKILSRDVDRVKRITMAIWNTMQFLKSCFQWESTLRSTIAFAVFLITVWNFELYMIPLALLLIFVYNFIRPVKGKVSSIQDSQESTDIDDEEDEDDKESEKKGLIERIYMVQDIVSTVQNVLEEIASFGERIKNTFNWTVPFLSSLACLILAAATIILYFIPLRYIILIWGINKFTKKLRNPYSIDNNELLDFLSRVPSDVQKVQYAELKLCSSHSPLRKKRSAL",
# MCTP1_HUMAN
"MEPRAAAAGEPEPPAASSSFQARLWKNLQLGVGRSKGGGGGRAGGPERRTADTPSPSPPPPVGTGNAPARGSGAGSRWSGFKKRKQVLDRVFSSSQPNLCCSSPEPLEPGGAGRAEQGSTLRRRIREHLLPAVKGPAAASGAAGGTPPGGRSPDSAPSSSSASSSLSSSPQPPPRGDRARDEGARRQGPGAHLCHQKSSSLPGTACLEQLLEPPPPPAEPARSPAESRAPETGEEHGSSQKIINTAGTSNAEVPLADPGMYQLDITLRRGQSLAARDRGGTSDPYVKFKIGGKEVFRSKIIHKNLNPVWEEKACILVDHLREPLYIKVFDYDFGLQDDFMGSAFLDLTQLELNRPTDVTLTLKDPHYPDHDLGIILLSVILTPKEGESRDVTMLMRKSWKRSSKELSENEVVGSYFSVKSLFWRTCGRPALPVLGFCRAELQNPYCKNVQFQTQSLRLSDLHRKSHLWRGIVSITLIEGRDLKAMDSNGLSDPYVKFRLGHQKYKSKIMPKTLNPQWREQFDFHLYEERGGVIDITAWDKDAGKRDDFIGRCQVDLSALSREQTHKLELQLEEGEGHLVLLVTLTASATVSISDLSVNSLEDQKEREEILKRYSPLRIFHNLKDVGFLQVKVIRAEGLMAADVTGKSDPFCVVELNNDRLLTHTVYKNLNPEWNKVFTFNIKDIHSVLEVTVYDEDRDRSADFLGKVAIPLLSIQNGEQKAYVLKNKQLTGPTKGVIYLEIDVIFNAVKASLRTLIPKEQKYIEEENRLSKQLLLRNFIRMKRCVMVLVNAAYYVNSCFDWDSPPRSLAAFVLFLFVVWNFELYMIPLVLLLLLTWNYFLIISGKDNRQRDTVVEDMLEDEEEEDDKDDKDSEKKGFINKIYAIQEVCVSVQNILDEVASFGERIKNTFNWTVPFLSWLAIVALCVFTAILYCIPLRYIVLVWGINKFTKKLRSPYAIDNNELLDFLSRVPSDVQVVQYQELKPDPSHSPYKRKKNNLG",
# IQEC1_HUMAN
"MWCLHCNSERTQSLLELELDSGVEGEAPSSETGTSLDSPSAYPQGPLVPGSSLSPDHYEHTSVGAYGLYSGPPGQQQRTRRPKLQHSTSILRKQAEEEAIKRSRSLSESYELSSDLQDKQVEMLERKYGGRLVTRHAARTIQTAFRQYQMNKNFERLRSSMSENRMSRRIVLSNMRMQFSFEGPEKVHSSYFEGKQVSVTNDGSQLGALVSPECGDLSEPTTLKSPAPSSDFADAITELEDAFSRQVKSLAESIDDALNCRSLHTEEAPALDAARARDTEPQTALHGMDHRKLDEMTASYSDVTLYIDEEELSPPLPLSQAGDRPSSTESDLRLRAGGAAPDYWALAHKEDKADTDTSCRSTPSLERQEQRLRVEHLPLLTIEPPSDSSVDLSDRSERGSLKRQSAYERSLGGQQGSPKHGPHSGAPKSLPREEPELRPRPPRPLDSHLAINGSANRQSKSESDYSDGDNDSINSTSNSNDTINCSSESSSRDSLREQTLSKQTYHKEARNSWDSPAFSNDVIRKRHYRIGLNLFNKKPEKGVQYLIERGFVPDTPVGVAHFLLQRKGLSRQMIGEFLGNRQKQFNRDVLDCVVDEMDFSTMELDEALRKFQAHIRVQGEAQKVERLIEAFSQRYCICNPGVVRQFRNPDTIFILAFAIILLNTDMYSPNVKPERKMKLEDFIKNLRGVDDGEDIPREMLMGIYERIRKRELKTNEDHVSQVQKVEKLIVGKKPIGSLHPGLGCVLSLPHRRLVCYCRLFEVPDPNKPQKLGLHQREIFLFNDLLVVTKIFQKKKNSVTYSFRQSFSLYGMQVLLFENQYYPNGIRLTSSVPGADIKVLINFNAPNPQDRKKFTDDLRESIAEVQEMEKHRIESELEKQKGVVRPSMSQCSSLKKESGNGTLSRACLDDSYASGEGLKRSALSSSLRDLSEAGKRGRRSSAGSLESNVEFQPFEPLQPSVLCS",
# NAPEP_HUMAN
"MDENESNQSLMTSSQYPKEAVRKRQNSARNSGASDSSRFSRKSFKLDYRLEEDVTKSKKGKDGRFVNPWPTWKNPSIPNVLRWLIMEKDHSSVPSSKEELDKELPVLKPYFITNPEEAGVREAGLRVTWLGHATVMVEMDELIFLTDPIFSSRASPSQYMGPKRFRRSPCTISELPPIDAVLISHNHYDHLDYNSVIALNERFGNELRWFVPLGLLDWMQKCGCENVIELDWWEENCVPGHDKVTFVFTPSQHWCKRTLMDDNKVLWGSWSVLGPWNRFFFAGDTGYCPAFEEIGKRFGPFDLAAIPIGAYEPRWFMKYQHVDPEEAVRIHTDVQTKKSMAIHWGTFALANEHYLEPPVKLNEALERYGLNAEDFFVLKHGESRYLNNDDENF",
# R7BP_HUMAN
"MSSAPNGRKKRPSRSTRSSIFQISKPPLQSGDWERRGSGSESAHKTQRALDDCKMLVQEFNTQVALYRELVISIGDVSVSCPSLRAEMHKTRTKGCEMARQAHQKLAAISGPEDGEIHPEICRLYIQLQCCLEMYTTEMLKSICLLGSLQFHRKGKEPGGGTKSLDCKIEESAETPALEDSSSSPVDSQQHSWQVSTDIENTERDMREMKNLLSKLRETMPLPLKNQDDSSLLNLTPYPLVRRRKRRFFGLCCLISS",
# TEN4_HUMAN
"MDVKERKPYRSLTRRRDAERRYTSSSADSEEGKAPQKSYSSSETLKAYDQDARLAYGSRVKDIVPQEAEEFCRTGANFTLRELGLEEVTPPHGTLYRTDIGLPHCGYSMGAGSDADMEADTVLSPEHPVRLWGRSTRSGRSSCLSSRANSNLTLTDTEHENTETDHPGGLQNHARLRTPPPPLSHAHTPNQHHAASINSLNRGNFTPRSNPSPAPTDHSLSGEPPAGGAQEPAHAQENWLLNSNIPLETRNLGKQPFLGTLQDNLIEMDILGASRHDGAYSDGHFLFKPGGTSPLFCTTSPGYPLTSSTVYSPPPRPLPRSTFARPAFNLKKPSKYCNWKCAALSAIVISATLVILLAYFVAMHLFGLNWHLQPMEGQMYEITEDTASSWPVPTDVSLYPSGGTGLETPDRKGKGTTEGKPSSFFPEDSFIDSGEIDVGRRASQKIPPGTFWRSQVFIDHPVHLKFNVSLGKAALVGIYGRKGLPPSHTQFDFVELLDGRRLLTQEARSLEGTPRQSRGTVPPSSHETGFIQYLDSGIWHLAFYNDGKESEVVSFLTTAIESVDNCPSNCYGNGDCISGTCHCFLGFLGPDCGRASCPVLCSGNGQYMKGRCLCHSGWKGAECDVPTNQCIDVACSNHGTCITGTCICNPGYKGESCEEVDCMDPTCSGRGVCVRGECHCSVGWGGTNCETPRATCLDQCSGHGTFLPDTGLCSCDPSWTGHDCSIEICAADCGGHGVCVGGTCRCEDGWMGAACDQRACHPRCAEHGTCRDGKCECSPGWNGEHCTIAHYLDRVVKEGCPGLCNGNGRCTLDLNGWHCVCQLGWRGAGCDTSMETACGDSKDNDGDGLVDCMDPDCCLQPLCHINPLCLGSPNPLDIIQETQVPVSQQNLHSFYDRIKFLVGRDSTHIIPGENPFDGGHACVIRGQVMTSDGTPLVGVNISFVNNPLFGYTISRQDGSFDLVTNGGISIILRFERAPFITQEHTLWLPWDRFFVMETIIMRHEENEIPSCDLSNFARPNPVVSPSPLTSFASSCAEKGPIVPEIQALQEEISISGCKMRLSYLSSRTPGYKSVLRISLTHPTIPFNLMKVHLMVAVEGRLFRKWFAAAPDLSYYFIWDKTDVYNQKVFGLSEAFVSVGYEYESCPDLILWEKRTTVLQGYEIDASKLGGWSLDKHHALNIQSGILHKGNGENQFVSQQPPVIGSIMGNGRRRSISCPSCNGLADGNKLLAPVALTCGSDGSLYVGDFNYIRRIFPSGNVTNILELRNKDFRHSHSPAHKYYLATDPMSGAVFLSDSNSRRVFKIKSTVVVKDLVKNSEVVAGTGDQCLPFDDTRCGDGGKATEATLTNPRGITVDKFGLIYFVDGTMIRRIDQNGIISTLLGSNDLTSARPLSCDSVMDISQVHLEWPTDLAINPMDNSLYVLDNNVVLQISENHQVRIVAGRPMHCQVPGIDHFLLSKVAIHATLESATALAVSHNGVLYIAETDEKKINRIRQVTTSGEISLVAGAPSGCDCKNDANCDCFSGDDGYAKDAKLNTPSSLAVCADGELYVADLGNIRIRFIRKNKPFLNTQNMYELSSPIDQELYLFDTTGKHLYTQSLPTGDYLYNFTYTGDGDITLITDNNGNMVNVRRDSTGMPLWLVVPDGQVYWVTMGTNSALKSVTTQGHELAMMTYHGNSGLLATKSNENGWTTFYEYDSFGRLTNVTFPTGQVSSFRSDTDSSVHVQVETSSKDDVTITTNLSASGAFYTLLQDQVRNSYYIGADGSLRLLLANGMEVALQTEPHLLAGTVNPTVGKRNVTLPIDNGLNLVEWRQRKEQARGQVTVFGRRLRVHNRNLLSLDFDRVTRTEKIYDDHRKFTLRILYDQAGRPSLWSPSSRLNGVNVTYSPGGYIAGIQRGIMSERMEYDQAGRITSRIFADGKTWSYTYLEKSMVLLLHSQRQYIFEFDKNDRLSSVTMPNVARQTLETIRSVGYYRNIYQPPEGNASVIQDFTEDGHLLHTFYLGTGRRVIYKYGKLSKLAETLYDTTKVSFTYDETAGMLKTINLQNEGFTCTIRYRQIGPLIDRQIFRFTEEGMVNARFDYNYDNSFRVTSMQAVINETPLPIDLYRYDDVSGKTEQFGKFGVIYYDINQIITTAVMTHTKHFDAYGRMKEVQYEIFRSLMYWMTVQYDNMGRVVKKELKVGPYANTTRYSYEYDADGQLQTVSINDKPLWRYSYDLNGNLHLLSPGNSARLTPLRYDIRDRITRLGDVQYKMDEDGFLRQRGGDIFEYNSAGLLIKAYNRAGSWSVRYRYDGLGRRVSSKSSHSHHLQFFYADLTNPTKVTHLYNHSSSEITSLYYDLQGHLFAMELSSGDEFYIACDNIGTPLAVFSGTGLMIKQILYTAYGEIYMDTNPNFQIIIGYHGGLYDPLTKLVHMGRRDYDVLAGRWTSPDHELWKHLSSSNVMPFNLYMFKNNNPISNSQDIKCFMTDVNSWLLTFGFQLHNVIPGYPKPDMDAMEPSYELIHTQMKTQEWDNSKSILGVQCEVQKQLKAFVTLERFDQLYGSTITSCQQAPKTKKFASSGSVFGKGVKFALKDGRVTTDIISVANEDGRRVAAILNHAHYLENLHFTIDGVDTHYFVKPGPSEGDLAILGLSGGRRTLENGVNVTVSQINTVLNGRTRRYTDIQLQYGALCLNTRYGTTLDEEKARVLELARQRAVRQAWAREQQRLREGEEGLRAWTEGEKQQVLSTGRVQGYDGFFVISVEQYPELSDSANNIHFMRQSEMGRR",
# C2D1A_HUMAN
"MHKRKGPPGPPGRGAAAARQLGLLVDLSPDGLMIPEDGANDEELEAEFLALVGGQPPALEKLKGKGPLPMEAIEKMASLCMRDPDEDEEEGTDEDDLEADDDLLAELNEVLGEEQKASETPPPVAQPKPEAPHPGLETTLQERLALYQTAIESARQAGDSAKMRRYDRGLKTLENLLASIRKGNAIDEADIPPPVAIGKGPASTPTYSPAPTQPAPRIASAPEPRVTLEGPSATAPASSPGLAKPQMPPGPCSPGPLAQLQSRQRDYKLAALHAKQQGDTTAAARHFRVAKSFDAVLEALSRGEPVDLSCLPPPPDQLPPDPPSPPSQPPTPATAPSTTEVPPPPRTLLEALEQRMERYQVAAAQAKSKGDQRKARMHERIVKQYQDAIRAHKAGRAVDVAELPVPPGFPPIQGLEATKPTQQSLVGVLETAMKLANQDEGPEDEEDEVPKKQNSPVAPTAQPKAPPSRTPQSGSAPTAKAPPKATSTRAQQQLAFLEGRKKQLLQAALRAKQKNDVEGAKMHLRQAKGLEPMLEASRNGLPVDITKVPPAPVNKDDFALVQRPGPGLSQEAARRYGELTKLIRQQHEMCLNHSNQFTQLGNITETTKFEKLAEDCKRSMDILKQAFVRGLPTPTARFEQRTFSVIKIFPDLSSNDMLLFIVKGINLPTPPGLSPGDLDVFVRFDFPYPNVEEAQKDKTSVIKNTDSPEFKEQFKLCINRSHRGFRRAIQTKGIKFEVVHKGGLFKTDRVLGTAQLKLDALEIACEVREILEVLDGRRPTGGRLEVMVRIREPLTAQQLETTTERWLVIDPVPAAVPTQVAGPKGKAPPVPAPARESGNRSARPLHSLSVLAFDQERLERKILALRQARRPVPPEVAQQYQDIMQRSQWQRAQLEQGGVGIRREYAAQLERQLQFYTEAARRLGNDGSRDAAKEALYRRNLVESELQRLRR",
# F171B_HUMAN
"MARLCRRVPCTLLLGLAVVLLKARLVPAAARAELSRSDLSLIQQQQQQQQQQQQQQKQLEEAEEERTEVPGATSTLTVPVSVFMLKVQVNDIISRQYLSQAVVEVFVNYTKTNSTVTKSNGAVLIKVPYKLGLSLTIIAYKDGYVLTPLPWKTRRMPIYSSVTLSLFPQSQANIWLFEDTVLITGKLADAKSQPSVQFSKALIKLPDNHHISNVTGYLTVLQQFLKVDNFLHTTGITLNKPGFENIELTPLAAICVKIYSGGKELKVNGSIQVSLPLLRLNDISAGDRIPAWTFDMNTGAWVNHGRGMVKEHNNHLIWTYDAPHLGYWIAAPLPGTRGSGINEDSKDITAYHTVFLTAILGGTIVIVIGFFAVLLCYCRDKCGTPQKRERNITKLEVLKRDQTTSTTHINHISTVKVALKAEDKSQLFNAKNSSYSPQKKEPSKAETEERVSMVKTRDDFKIYNEDVSFLSVNQNNYSRNPTQSLEPNVGSKQPKHINNNLSSSLGDAQDEKRYLTGNEEAYGRSHIPEQLMHIYSQPIAILQTSDLFSTPEQLHTAKSATLPRKGQLVYGQLMEPVNRENFTQTLPKMPIHSHAQPPDAREEDIILEGQQSLPSQASDWSRYSSSLLESVSVPGTLNEAVVMTPFSSELQGISEQTLLELSKGKPSPHPRAWFVSLDGKPVAQVRHSFIDLKKGKRTQSNDTSLDSGVDMNELHSSRKLEREKTFIKSMHQPKILYLEDLDLSSSESGTTVCSPEDPALRHILDGGSGVIMEHPGEESPGRKSTVEDFEANTSPTKRRGRPPLAKRDSKTNIWKKREERPLIPIN",
# CNKR3_HUMAN
"MEPVTKWSPKQVVDWTRGLDDCLQQYVHKFEREKINGEQLLQISHQDLEELGVTRIGHQELVLEAVDLLCALNYGLETDNMKNLVLKLRASSHNLQNYISSRRKSPAYDGNTSRKAPNEFLTSVVELIGAAKALLAWLDRAPFTGITDFSVTKNKIIQLCLDLTTTVQKDCFVAEMEDKVLTVVKVLNGICDKTIRSTTDPVMSQCACLEEVHLPNIKPGEGLGMYIKSTYDGLHVITGTTENSPADRSQKIHAGDEVIQVNQQTVVGWQLKNLVKKLRENPTGVVLLLKKRPTGSFNFTPAPLKNLRWKPPLVQTSPPPATTQSPESTMDTSLKKEKSAILDLYIPPPPAVPYSPRDENGSFVYGGSSKCKQPLPGPKGSESPNSFLDQESRRRRFTIADSDQLPGYSVETNILPTKMREKTPSYGKPRPLSMPADGNWMGIVDPFARPRGHGRKGEDALCRYFSNERIPPIIEESSSPPYRFSRPTTERHLVRGADYIRGSRCYINSDLHSSATIPFQEEGTKKKSGSSATKSSSTEPSLLVSWFTRLKLLTH",
# AB17C_HUMAN
"MPEPGPRMNGFSLGELCWLFCCPPCPSRIAAKLAFLPPEPTYTVLAPEQRGAGASAPAPAQATAAAAAAQPAPQQPEEGAGAGPGACSLHLSERADWQYSQRELDAVEVFFSRTARDNRLGCMFVRCAPSSRYTLLFSHGNAVDLGQMCSFYIGLGSRINCNIFSYDYSGYGVSSGKPSEKNLYADIDAAWQALRTRYGVSPENIILYGQSIGTVPTVDLASRYECAAVILHSPLMSGLRVAFPDTRKTYCFDAFPSIDKISKVTSPVLVIHGTEDEVIDFSHGLAMYERCPRAVEPLWVEGAGHNDIELYAQYLERLKQFISHELPNS",
# CNIH2_HUMAN
"MAFTFAAFCYMLTLVLCASLIFFVIWHIIAFDELRTDFKNPIDQGNPARARERLKNIERICCLLRKLVVPEYSIHGLFCLMFLCAAEWVTLGLNIPLLFYHLWRYFHRPADGSEVMYDAVSIMNADILNYCQKESWCKLAFYLLSFFYYLYSMVYTLVSF",
# GLD2_HUMAN
"MFPNSILGRPPFTPNHQQHNNFFTLSPTVYSHQQLIDAQFNFQNADLSRAVSLQQLTYGNVSPIQTSASPLFRGRKRLSDEKNLPLDGKRQRFHSPHQEPTVVNQIVPLSGERRYSMPPLFHTHYVPDIVRCVPPFREIAFLEPREITLPEAKDKLSQQILELFETCQQQISDLKKKELCRTQLQREIQLLFPQSRLFLVGSSLNGFGTRSSDGDLCLVVKEEPCFFQVNQKTEARHILTLVHKHFCTRLSGYIERPQLIRAKVPIVKFRDKVSCVEFDLNVNNIVGIRNTFLLRTYAYLENRVRPLVLVIKKWASHHQINDASRGTLSSYSLVLMVLHYLQTLPEPILPSLQKIYPESFSPAIQLHLVHQAPCNVPPYLSKNESNLGDLLLGFLKYYATEFDWNSQMISVREAKAIPRPDGIEWRNKYICVEEPFDGTNTARAVHEKQKFDMIKDQFLKSWHRLKNKRDLNSILPVRAAVLKR",
# LRFN4_HUMAN
"MAPPLLLLLLASGAAACPLPCVCQNLSESLSTLCAHRGLLFVPPNVDRRTVELRLADNFIQALGPPDFRNMTGLVDLTLSRNAITRIGARAFGDLESLRSLHLDGNRLVELGTGSLRGPVNLQHLILSGNQLGRIAPGAFDDFLESLEDLDLSYNNLRQVPWAGIGAMPALHTLNLDHNLIDALPPGAFAQLGQLSRLDLTSNRLATLAPDPLFSRGRDAEASPAPLVLSFSGNPLHCNCELLWLRRLARPDDLETCASPPGLAGRYFWAVPEGEFSCEPPLIARHTQRLWVLEGQRATLRCRALGDPAPTMHWVGPDDRLVGNSSRARAFPNGTLEIGVTGAGDAGGYTCIATNPAGEATARVELRVLALPHGGNSSAEGGRPGPSDIAASARTAAEGEGTLESEPAVQVTEVTATSGLVSWGPGRPADPVWMFQIQYNSSEDETLIYRIVPASSHHFLLKHLVPGADYDLCLLALSPAAGPSDLTATRLLGCAHFSTLPASPLCHALQAHVLGGTLTVAVGGVLVAALLVFTVALLVRGRGAGNGRLPLKLSHVQSQTNGGPSPTPKAHPPRSPPPRPQRSCSLDLGDAGCYGYARRLGGAWARRSHSVHGGLLGAGCRGVGGSAERLEESVV",
# CPLX2_HUMAN
"MDFVMKQALGGATKDMGKMLGGEEEKDPDAQKKEEERQEALRQQEEERKAKHARMEAEREKVRQQIRDKYGLKKKEEKEAEEKAALEQPCEGSLTRPKKAIPAGCGDEEEEEEESILDTVLKYLPGPLQDMFKK",
# SARM1_HUMAN
"MVLTLLLSAYKLCRFFAMSGPRPGAERLAVPGPDGGGGTGPWWAAGGRGPREVSPGAGTEVQDALERALPELQQALSALKQAGGARAVGAGLAEVFQLVEEAWLLPAVGREVAQGLCDAIRLDGGLDLLLRLLQAPELETRVQAARLLEQILVAENRDRVARIGLGVILNLAKEREPVELARSVAGILEHMFKHSEETCQRLVAAGGLDAVLYWCRRTDPALLRHCALALGNCALHGGQAVQRRMVEKRAAEWLFPLAFSKEDELLRLHACLAVAVLATNKEVEREVERSGTLALVEPLVASLDPGRFARCLVDASDTSQGRGPDDLQRLVPLLDSNRLEAQCIGAFYLCAEAAIKSLQGKTKVFSDIGAIQSLKRLVSYSTNGTKSALAKRALRLLGEEVPRPILPSVPSWKEAEVQTWLQQIGFSKYCESFREQQVDGDLLLRLTEEELQTDLGMKSGITRKRFFRELTELKTFANYSTCDRSNLADWLGSLDPRFRQYTYGLVSCGLDRSLLHRVSEQQLLEDCGIHLGVHRARILTAAREMLHSPLPCTGGKPSGDTPDVFISYRRNSGSQLASLLKVHLQLHGFSVFIDVEKLEAGKFEDKLIQSVMGARNFVLVLSPGALDKCMQDHDCKDWVHKEIVTALSCGKNIVPIIDGFEWPEPQVLPEDMQAVLTFNGIKWSHEYQEATIEKIIRFLQGRSSRDSSAGSDTSLEGAAPMGPT",
# KLH17_HUMAN
"MQPRSERPAGRTQSPEHGSPGPGPEAPPPPPPQPPAPEAERTRPRQARPAAPMEGAVQLLSREGHSVAHNSKRHYHDAFVAMSRMRQRGLLCDIVLHVAAKEIRAHKVVLASCSPYFHAMFTNEMSESRQTHVTLHDIDPQALDQLVQFAYTAEIVVGEGNVQTLLPAASLLQLNGVRDACCKFLLSQLDPSNCLGIRGFADAHSCSDLLKAAHRYVLQHFVDVAKTEEFMLLPLKQVLELVSSDSLNVPSEEEVYRAVLSWVKHDVDARRQHVPRLMKCVRLPLLSRDFLLGHVDAESLVRHHPDCKDLLIEALKFHLLPEQRGVLGTSRTRPRRCEGAGPVLFAVGGGSLFAIHGDCEAYDTRTDRWHVVASMSTRRARVGVAAVGNRLYAVGGYDGTSDLATVESYDPVTNTWQPEVSMGTRRSCLGVAALHGLLYSAGGYDGASCLNSAERYDPLTGTWTSVAAMSTRRRYVRVATLDGNLYAVGGYDSSSHLATVEKYEPQVNVWSPVASMLSRRSSAGVAVLEGALYVAGGNDGTSCLNSVERYSPKAGAWESVAPMNIRRSTHDLVAMDGWLYAVGGNDGSSSLNSIEKYNPRTNKWVAASCMFTRRSSVGVAVLELLNFPPPSSPTLSVSSTSL",
# CRTC1_HUMAN
"MATSNNPRKFSEKIALHNQKQAEETAAFEEVMKDLSLTRAARLQLQKSQYLQLGPSRGQYYGGSLPNVNQIGSGTMDLPFQTPFQSSGLDTSRTTRHHGLVDRVYRERGRLGSPHRRPLSVDKHGRQADSCPYGTMYLSPPADTSWRRTNSDSALHQSTMTPTQPESFSSGSQDVHQKRVLLLTVPGMEETTSEADKNLSKQAWDTKKTGSRPKSCEVPGINIFPSADQENTTALIPATHNTGGSLPDLTNIHFPSPLPTPLDPEEPTFPALSSSSSTGNLAANLTHLGIGGAGQGMSTPGSSPQHRPAGVSPLSLSTEARRQQASPTLSPLSPITQAVAMDALSLEQQLPYAFFTQAGSQQPPPQPQPPPPPPPASQQPPPPPPPQAPVRLPPGGPLLPSASLTRGPQPPPLAVTVPSSLPQSPPENPGQPSMGIDIASAPALQQYRTSAGSPANQSPTSPVSNQGFSPGSSPQHTSTLGSVFGDAYYEQQMAARQANALSHQLEQFNMMENAISSSSLYSPGSTLNYSQAAMMGLTGSHGSLPDSQQLGYASHSGIPNIILTVTGESPPSLSKELTSSLAGVGDVSFDSDSQFPLDELKIDPLTLDGLHMLNDPDMVLADPATEDTFRMDRL",
# CBLN3_HUMAN
"MLGAKPHWLPGPLHSPGLPLVLVLLALGAGWAQEGSEPVLLEGECLVVCEPGRAAAGGPGGAALGEAPPGRVAFAAVRSHHHEPAGETGNGTSGAIYFDQVLVNEGGGFDRASGSFVAPVRGVYSFRFHVVKVYNRQTVQVSLMLNTWPVISAFANDPDVTREAATSSVLLPLDPGDRVSLRLRRGNLLGGWKYSSFSGFLIFPL",
# TM108_HUMAN
"MKRSLQALYCQLLSFLLILALTEALAFAIQEPSPRESLQVLPSGTPPGTMVTAPHSSTRHTSVVMLTPNPDGPPSQAAAPMATPTPRAEGHPPTHTISTIAATVTAPHSESSLSTGPAPAAMATTSSKPEGRPRGQAAPTILLTKPPGATSRPTTAPPRTTTRRPPRPPGSSRKGAGNSSRPVPPAPGGHSRSKEGQRGRNPSSTPLGQKRPLGKIFQIYKGNFTGSVEPEPSTLTPRTPLWGYSSSPQPQTVAATTVPSNTSWAPTTTSLGPAKDKPGLRRAAQGGGSTFTSQGGTPDATAASGAPVSPQAAPVPSQRPHHGDPQDGPSHSDSWLTVTPGTSRPLSTSSGVFTAATGPTPAAFDTSVSAPSQGIPQGASTTPQAPTHPSRVSESTISGAKEETVATLTMTDRVPSPLSTVVSTATGNFLNRLVPAGTWKPGTAGNISHVAEGDKPQHRATICLSKMDIAWVILAISVPISSCSVLLTVCCMKRKKKTANPENNLSYWNNTITMDYFNRHAVELPREIQSLETSEDQLSEPRSPANGDYRDTGMVLVNPFCQETLFVGNDQVSEI",
# GSG1L_HUMAN
"MKTSRRGRALLAVALNLLALLFATTAFLTTHWCQGTQRVPKPGCGQGGRANCPNSGANATANGTAAPAAAAAAATASGNGPPGGALYSWETGDDRFLFRNFHTGIWYSCEEELSGLGEKCRSFIDLAPASEKGVLWLSVVSEVLYILLLVVGFSLMCLELFHSSNVIDGLKLNAFAAVFTVLSGLLGMVAHMMYTQVFQVTVSLGPEDWRPHSWDYGWSFCLAWGSFTCCMAASVTTLNSYTKTVIEFRHKRKVFEQGYREEPTFIDPEAIKYFRERMEKRDGSEEDFHLDCRHERYPARHQPHMADSWPRSSAQEAPELNRQCWVLGHWV",
# NSMF_HUMAN
"MGAAASRRRALRSEAMSSVAAKVRAARAFGEYLSQSHPENRNGADHLLADAYSGHDGSPEMQPAPQNKRRLSLVSNGCYEGSLSEEPSIRKPAGEGPQPRVYTISGEPALLPSPEAEAIELAVVKGRRQRHPHHHSQPLRASPGGSREDVSRPCQSWAGSRQGSKECPGCAQLAPGPTPRAFGLDQPPLPETSGRRKKLERMYSVDRVSDDIPIRTWFPKENLFSFQTATTTMQAISVFRGYAERKRRKRENDSASVIQRNFRKHLRMVGSRRVKAQTFAERRERSFSRSWSDPTPMKADTSHDSRDSSDLQSSHCTLDEAFEDLDWDTEKGLEAVACDTEGFVPPKVMLISSKVPKAEYIPTIIRRDDPSIIPILYDHEHATFEDILEEIERKLNVYHKGAKIWKMLIFCQGGPGHLYLLKNKVATFAKVEKEEDMIHFWKRLSRLMSKVNPEPNVIHIMGCYILGNPNGEKLFQNLRTLMTPYRVTFESPLELSAQGKQMIETYFDFRLYRLWKSRQHSKLLDFDDVL",
# SYT10_HUMAN
"MSFHKEDGVNSLCQKALHIVTELCFAGQVEWEKCSGIFPRDRGSQGGSSTDISVSLLAVVVSFCGLALLVVSLFVFWKLCWPCWKSKPVTSNITTLPQSISSAPTEVFETEEKKEIKENEKPAVKAIEPAIKISHTSPDIPAEVQTALKEHLIKHARVQRQITEPTSSTRHSSFRRHLPRQMQVSSVDFSMGTEPVLQRGETTTSIGRIKPELYKQKSVDSEGNQNEDVKICGKLNFTLQYDYENELLVVKIIKALDLPAKDFTGTSDPYVKMYLLPDRKKKFQTRVHRKTLNPLFDETFQFPVAYDQLSNRKLHFSVYDFDRFSRHDMIGEVILDNLFEVSDLSREATVWKDIHCATTESIDLGEIMFSLCYLPTAGRMTLTVIKCRNLKAMDITGSSDPYVKVSLMCEGRRLKKRKTTTKKNTLNPVYNEAIIFDIPPENVDQVSLSIAVMDYDRVGHNEVIGVCRTGLDAEGLGRDHWNEMLAYHRKPITHWHPLLELPGRATSFDSQGSCPSPKPPSTP",
# DNMBP_HUMAN
"MEAGSVVRAIFDFCPSVSEELPLFVGDIIEVLAVVDEFWLLGKKEDVTGQFPSSFVEIVTIPSLKEGERLFVCICEFTSQELDNLPLHRGDLVILDGIPTAGWLQGRSCWGARGFFPSSCVRELCLSSQSRQWHSQSALFQIPEYSMGQARALMGLSAQLDEELDFREGDVITIIGVPEPGWFEGELEGRRGIFPEGFVELLGPLRTVDESVSSGNQDDCIVNGEVDTPVGEEEIGPDEDEEEPGTYGVALYRFQALEPNELDFEVGDKIRILATLEDGWLEGSLKGRTGIFPYRFVKLCPDTRVEETMALPQEGSLARIPETSLDCLENTLGVEEQRHETSDHEAEEPDCIISEAPTSPLGHLTSEYDTDRNSYQDEDTAGGPPRSPGVEWEMPLATDSPTSDPTEVVNGISSQPQVPFHPNLQKSQYYSTVGGSHPHSEQYPDLLPLEARTRDYASLPPKRMYSQLKTLQKPVLPLYRGSSVSASRVVKPRQSSPQLHNLASYTKKHHTSSVYSISERLEMKPGPQAQGLVMEAATHSQGDGSTDLDSKLTQQLIEFEKSLAGPGTEPDKILRHFSIMDFNSEKDIVRGSSKLITEQELPERRKALRPPPPRPCTPVSTSPHLLVDQNLKPAPPLVVRPSRPAPLPPSAQQRTNAVSPKLLSRHRPTCETLEKEGPGHMGRSLDQTSPCPLVLVRIEEMERDLDMYSRAQEELNLMLEEKQDESSRAETLEDLKFCESNIESLNMELQQLREMTLLSSQSSSLVAPSGSVSAENPEQRMLEKRAKVIEELLQTERDYIRDLEMCIERIMVPMQQAQVPNIDFEGLFGNMQMVIKVSKQLLAALEISDAVGPVFLGHRDELEGTYKIYCQNHDEAIALLEIYEKDEKIQKHLQDSLADLKSLYNEWGCTNYINLGSFLIKPVQRVMRYPLLLMELLNSTPESHPDKVPLTNAVLAVKEINVNINEYKRRKDLVLKYRKGDEDSLMEKISKLNIHSIIKKSNRVSSHLKHLTGFAPQIKDEVFEETEKNFRMQERLIKSFIRDLSLYLQHIRESACVKVVAAVSMWDVCMERGHRDLEQFERVHRYISDQLFTNFKERTERLVISPLNQLLSMFTGPHKLVQKRFDKLLDFYNCTERAEKLKDKKTLEELQSARNNYEALNAQLLDELPKFHQYAQGLFTNCVHGYAEAHCDFVHQALEQLKPLLSLLKVAGREGNLIAIFHEEHSRVLQQLQVFTFFPESLPATKKPFERKTIDRQSARKPLLGLPSYMLQSEELRASLLARYPPEKLFQAERNFNAAQDLDVSLLEGDLVGVIKKKDPMGSQNRWLIDNGVTKGFVYSSFLKPYNPRRSHSDASVGSHSSTESEHGSSSPRFPRQNSGSTLTFNPSSMAVSFTSGSCQKQPQDASPPPKECDQGTLSASLNPSNSESSPSRCPSDPDSTSQPRSGDSADVARDVKQPTATPRSYRNFRHPEIVGYSVPGRNGQSQDLVKGCARTAQAPEDRSTEPDGSEAEGNQVYFAVYTFKARNPNELSVSANQKLKILEFKDVTGNTEWWLAEVNGKKGYVPSNYIRKTEYT",
# INSY2_HUMAN
"MVSKDTGKCILTTSESEVEPAACLALEMKYALDPNRQIKKRNKALQVRFKDICEAQNEQRDTQLSSGQLGEKREAKPVSCRAAYRKYMTVPARRSIPNVTKSTGVQTSPDLKKCYQTFPLDRKKGNLKSLPAADPFKSQNNGFLTDAKEKNEAGPMEEARPCGAGRVHKTTALVFHSNQHMNTVDQPLGVNCTEPCKSPEPLSYGEAALQNSTRPPSEEPDYQLLGRAKQDRGRPNSEEPAPPALRRVFKTEVATVYAPALSARAPEPGLSDSAAASQWSLCPADDERRRATHLNGLQAPSETALACSPPMQCLSPECSEQPSQTHTPPGLGNQPSPTAVAAGEECQRIVPHTEVVDLKAQLQMMENLISSSQETIKVLLGVIQELEKGEAHREGLSYRTGQDTANCDTCRNSACIIYSVELDFKQQEDKLQPVLRKLHPIEETQVIPSPYSQETYSSTPKQKSKTESKKHGRWKLWFL",
# SHSA6_HUMAN
"MALRRLLLLLLLSLESLDLLPSVHGARGRAANRTLSAGGAAVGGRRAGGALARGGRELNGTARAPGIPEAGSRRGQPAAAVAAAASAAVTYETCWGYYDVSGQYDKEFECNNSESGYLYCCGTCYYRFCCKKRHEKLDQRQCTNYQSPVWVQTPSTKVVSPGPENKYDPEKDKTNFTVYITCGVIAFVIVAGVFAKVSYDKAHRPPREMNIHRALADILRQQGPIPIAHCERETISAIDTSPKENTPVRSSSKNHYTPVRTAKQTPEKPRMNNILTSATEPYDLSFSRSFQNLAHLPPSYESAVKTNPSKYSSLKRLTDKEADEYYMRRRHLPDLAARGTLPLNVIQMSQQKPLPRERPRRPIRAMSQDRVLSPDRGLPDEFSMPYDRILSDEQLLSTERLHSQDPLLSPERTAFPEQSLSRAISHTDVFVSTPVLDRYRMSKMHSHPSASNNSYATLGQSQTAAKRHAFASRRHNTVEQLHYIPGHHTCYTASKTEVTV",
# SYDE1_HUMAN
"MAEPLLRKTFSRLRGREKLPRKKSDAKERGHPAQRPEPSPPEPEPQAPEGSQAGAEGPSSPEASRSPARGAYLQSLEPSSRRWVLGGAKPAEDTSLGPGVPGTGEPAGEIWYNPIPEEDPRPPAPEPPGPQPGSAESEGLAPQGAAPASPPTKASRTKSPGPARRLSIKMKKLPELRRRLSLRGPRAGRERERAAPAGSVISRYHLDSSVGGPGPAAGPGGTRSPRAGYLSDGDSPERPAGPPSPTSFRPYEVGPAARAPPAALWGRLSLHLYGLGGLRPAPGATPRDLCCLLQVDGEARARTGPLRGGPDFLRLDHTFHLELEAARLLRALVLAWDPGVRRHRPCAQGTVLLPTVFRGCQAQQLAVRLEPQGLLYAKLTLSEQQEAPATAEPRVFGLPLPLLVERERPPGQVPLIIQKCVGQIERRGLRVVGLYRLCGSAAVKKELRDAFERDSAAVCLSEDLYPDINVITGILKDYLRELPTPLITQPLYKVVLEAMARDPPNRVPPTTEGTRGLLSCLPDVERATLTLLLDHLRLVSSFHAYNRMTPQNLAVCFGPVLLPARQAPTRPRARSSGPGLASAVDFKHHIEVLHYLLQSWPDPRLPRQSPDVAPYLRPKRQPPLHLPLADPEVVTRPRGRGGPESPPSNRYAGDWSVCGRDFLPCGRDFLSGPDYDHVTGSDSEDEDEEVGEPRVTGDFEDDFDAPFNPHLNLKDFDALILDLERELSKQINVCL",
# KCTD8_HUMAN
"MALKDTGSGGSTILPISEMVSSSSSPGASAAAAPGPCAPSPFPEVVELNVGGQVYVTKHSTLLSVPDSTLASMFSPSSPRGGARRRGELPRDSRARFFIDRDGFLFRYVLDYLRDKQLALPEHFPEKERLLREAEYFQLTDLVKLLSPKVTKQNSLNDEGCQSDLEDNVSQGSSDALLLRGAAAAVPSGPGAHGGGGGGGAQDKRSGFLTLGYRGSYTTVRDNQADAKFRRVARIMVCGRIALAKEVFGDTLNESRDPDRQPEKYTSRFYLKFTYLEQAFDRLSEAGFHMVACNSSGTAAFVNQYRDDKIWSSYTEYIFFRPPQKIVSPKQEHEDRKHDKVTDKGSESGTSCNELSTSSCDSHSEASTPQDNPSSAQQATAHQPNTLTLDRPSKKAPVQWIPPPDKRRNSELFQTLISKSRETNLSKKKVCEKLSVEEEMKKCIQDFKKIHIPDYFPERKRQWQSELLQKYGL",
# RAPH1_HUMAN
"MEQLSDEEIDHGAEEDSDKEDQDLDKMFGAWLGELDKLTQSLDSDKPMEPVKRSPLRQETNMANFSYRFSIYNLNEALNQGETVDLDALMADLCSIEQELSSIGSGNSKRQITETKATQKLPVSRHTLKHGTLKGLSSSSNRIAKPSHASYSLDDVTAQLEQASLSMDEAAQQSVLEDTKPLVTNQHRRTASAGTVSDAEVHSISNSSHSSITSAASSMDSLDIDKVTRPQELDLTHQGQPITEEEQAAKLKAEKIRVALEKIKEAQVKKLVIRVHMSDDSSKTMMVDERQTVRQVLDNLMDKSHCGYSLDWSLVETVSELQMERIFEDHENLVENLLNWTRDSQNKLIFMERIEKYALFKNPQNYLLGKKETAEMADRNKEVLLEECFCGSSVTVPEIEGVLWLKDDGKKSWKKRYFLLRASGIYYVPKGKAKVSRDLVCFLQLDHVNVYYGQDYRNKYKAPTDYCLVLKHPQIQKKSQYIKYLCCDDVRTLHQWVNGIRIAKYGKQLYMNYQEALKRTESAYDWTSLSSSSIKSGSSSSSIPESQSNHSNQSDSGVSDTQPAGHVRSQSIVSSVFSEAWKRGTQLEESSKARMESMNRPYTSLVPPLSPQPKIVTPYTASQPSPPLPPPPPPPPPPPPPPPPPPPPLPSQSAPSAGSAAPMFVKYSTITRLQNASQHSGALFKPPTPPVMQSQSVKPQILVPPNGVVPPPPPPPPPPTPGSAMAQLKPAPCAPSLPQFSAPPPPLKIHQVQHITQVAPPTPPPPPPIPAPLPPQAPPKPLVTIPAPTSTKTVAPVVTQAAPPTPTPPVPPAKKQPAFPASYIPPSPPTPPVPVPPPTLPKQQSFCAKPPPSPLSPVPSVVKQIASQFPPPPTPPAMESQPLKPVPANVAPQSPPAVKAKPKWQPSSIPVPSPDFPPPPPESSLVFPPPPPSPVPAPPPPPPPTASPTPDKSGSPGKKTSKTSSPGGKKPPPTPQRNSSIKSSSGAEHPEPKRPSVDSLVSKFTPPAESGSPSKETLPPPAAPPKPGKLNLSGVNLPGVLQQGCVSAKAPVLSGRGKDSVVEFPSPPSDSDFPPPPPETELPLPPIEIPAVFSGNTSPKVAVVNPQPQQWSKMSVKKAPPPTRPKRNDSTRLTQAEISEQPTMATVVPQVPTSPKSSLSVQPGFLADLNRTLQRKSITRHGSLSSRMSRAEPTATMDDMALPPPPPELLSDQQKAGYGGSHISGYATLRRGPPPAPPKRDQNTKLSRDW",
# TBA1A_HUMAN
"MRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPSDKTIGGGDDSFNTFFSETGAGKHVPRAVFVDLEPTVIDEVRTGTYRQLFHPEQLITGKEDAANNYARGHYTIGKEIIDLVLDRIRKLADQCTGLQGFLVFHSFGGGTGSGFTSLLMERLSVDYGKKSKLEFSIYPAPQVSTAVVEPYNSILTTHTTLEHSDCAFMVDNEAIYDICRRNLDIERPTYTNLNRLIGQIVSSITASLRFDGALNVDLTEFQTNLVPYPRIHFPLATYAPVISAEKAYHEQLSVAEITNACFEPANQMVKCDPRHGKYMACCLLYRGDVVPKDVNAAIATIKTKRTIQFVDWCPTGFKVGINYQPPTVVPGGDLAKVQRAVCMLSNTTAIAEAWARLDHKFDLMYAKRAFVHWYVGEGMEEGEFSEAREDMAALEKDYEEVGVDSVEGEGEEEGEEY",
# MARK2_HUMAN
"MSSARTPLPTLNERDTEQPTLGHLDSKPSSKSNMIRGRNSATSADEQPHIGNYRLLKTIGKGNFAKVKLARHILTGKEVAVKIIDKTQLNSSSLQKLFREVRIMKVLNHPNIVKLFEVIETEKTLYLVMEYASGGEVFDYLVAHGRMKEKEARAKFRQIVSAVQYCHQKFIVHRDLKAENLLLDADMNIKIADFGFSNEFTFGNKLDTFCGSPPYAAPELFQGKKYDGPEVDVWSLGVILYTLVSGSLPFDGQNLKELRERVLRGKYRIPFYMSTDCENLLKKFLILNPSKRGTLEQIMKDRWMNVGHEDDELKPYVEPLPDYKDPRRTELMVSMGYTREEIQDSLVGQRYNEVMATYLLLGYKSSELEGDTITLKPRPSADLTNSSAPSPSHKVQRSVSANPKQRRFSDQAAGPAIPTSNSYSKKTQSNNAENKRPEEDRESGRKASSTAKVPASPLPGLERKKTTPTPSTNSVLSTSTNRSRNSPLLERASLGQASIQNGKDSLTMPGSRASTASASAAVSAARPRQHQKSMSASVHPNKASGLPPTESNCEVPRPSTAPQRVPVASPSAHNISSSGGAPDRTNFPRGVSSRSTFHAGQLRQVRDQQNLPYGVTPASPSGHSQGRRGASGSIFSKFTSKFVRRNLSFRFARRNLNEPESKDRVETLRPHVVGSGGNDKEKEEFREAKPRSLRFTWSMKTTSSMEPNEMMREIRKVLDANSCQSELHEKYMLLCMHGTPGHEDFVQWEMEVCKLPRLSLNGVRFKRISGTSMAFKNIASKIANELKL",
# SV2A_HUMAN
"MEEGFRDRAAFIRGAKDIAKEVKKHAAKKVVKGLDRVQDEYSRRSYSRFEEEDDDDDFPAPSDGYYRGEGTQDEEEGGASSDATEGHDEDDEIYEGEYQGIPRAESGGKGERMADGAPLAGVRGGLSDGEGPPGGRGEAQRRKEREELAQQYEAILRECGHGRFQWTLYFVLGLALMADGVEVFVVGFVLPSAEKDMCLSDSNKGMLGLIVYLGMMVGAFLWGGLADRLGRRQCLLISLSVNSVFAFFSSFVQGYGTFLFCRLLSGVGIGGSIPIVFSYFSEFLAQEKRGEHLSWLCMFWMIGGVYAAAMAWAIIPHYGWSFQMGSAYQFHSWRVFVLVCAFPSVFAIGALTTQPESPRFFLENGKHDEAWMVLKQVHDTNMRAKGHPERVFSVTHIKTIHQEDELIEIQSDTGTWYQRWGVRALSLGGQVWGNFLSCFGPEYRRITLMMMGVWFTMSFSYYGLTVWFPDMIRHLQAVDYASRTKVFPGERVEHVTFNFTLENQIHRGGQYFNDKFIGLRLKSVSFEDSLFEECYFEDVTSSNTFFRNCTFINTVFYNTDLFEYKFVNSRLINSTFLHNKEGCPLDVTGTGEGAYMVYFVSFLGTLAVLPGNIVSALLMDKIGRLRMLAGSSVMSCVSCFFLSFGNSESAMIALLCLFGGVSIASWNALDVLTVELYPSDKRTTAFGFLNALCKLAAVLGISIFTSFVGITKAAPILFASAALALGSSLALKLPETRGQVLQ",
# SV2B_HUMAN
"MDDYKYQDNYGGYAPSDGYYRGNESNPEEDAQSDVTEGHDEEDEIYEGEYQGIPHPDDVKAKQAKMAPSRMDSLRGQTDLMAERLEDEEQLAHQYETIMDECGHGRFQWILFFVLGLALMADGVEVFVVSFALPSAEKDMCLSSSKKGMLGMIVYLGMMAGAFILGGLADKLGRKRVLSMSLAVNASFASLSSFVQGYGAFLFCRLISGIGIGGALPIVFAYFSEFLSREKRGEHLSWLGIFWMTGGLYASAMAWSIIPHYGWGFSMGTNYHFHSWRVFVIVCALPCTVSMVALKFMPESPRFLLEMGKHDEAWMILKQVHDTNMRAKGTPEKVFTVSNIKTPKQMDEFIEIQSSTGTWYQRWLVRFKTIFKQVWDNALYCVMGPYRMNTLILAVVWFAMAFSYYGLTVWFPDMIRYFQDEEYKSKMKVFFGEHVYGATINFTMENQIHQHGKLVNDKFTRMYFKHVLFEDTFFDECYFEDVTSTDTYFKNCTIESTIFYNTDLYEHKFINCRFINSTFLEQKEGCHMDLEQDNDFLIYLVSFLGSLSVLPGNIISALLMDRIGRLKMIGGSMLISAVCCFFLFFGNSESAMIGWQCLFCGTSIAAWNALDVITVELYPTNQRATAFGILNGLCKFGAILGNTIFASFVGITKVVPILLAAASLVGGGLIALRLPETREQVLM",
# CYFP1_HUMAN
"MAAQVTLEDALSNVDLLEELPLPDQQPCIEPPPSSLLYQPNFNTNFEDRNAFVTGIARYIEQATVHSSMNEMLEEGQEYAVMLYTWRSCSRAIPQVKCNEQPNRVEIYEKTVEVLEPEVTKLMNFMYFQRNAIERFCGEVRRLCHAERRKDFVSEAYLITLGKFINMFAVLDELKNMKCSVKNDHSAYKRAAQFLRKMADPQSIQESQNLSMFLANHNKITQSLQQQLEVISGYEELLADIVNLCVDYYENRMYLTPSEKHMLLKVMGFGLYLMDGSVSNIYKLDAKKRINLSKIDKYFKQLQVVPLFGDMQIELARYIKTSAHYEENKSRWTCTSSGSSPQYNICEQMIQIREDHMRFISELARYSNSEVVTGSGRQEAQKTDAEYRKLFDLALQGLQLLSQWSAHVMEVYSWKLVHPTDKYSNKDCPDSAEEYERATRYNYTSEEKFALVEVIAMIKGLQVLMGRMESVFNHAIRHTVYAALQDFSQVTLREPLRQAIKKKKNVIQSVLQAIRKTVCDWETGHEPFNDPALRGEKDPKSGFDIKVPRRAVGPSSTQLYMVRTMLESLIADKSGSKKTLRSSLEGPTILDIEKFHRESFFYTHLINFSETLQQCCDLSQLWFREFFLELTMGRRIQFPIEMSMPWILTDHILETKEASMMEYVLYSLDLYNDSAHYALTRFNKQFLYDEIEAEVNLCFDQFVYKLADQIFAYYKVMAGSLLLDKRLRSECKNQGATIHLPPSNRYETLLKQRHVQLLGRSIDLNRLITQRVSAAMYKSLELAIGRFESEDLTSIVELDGLLEINRMTHKLLSRYLTLDGFDAMFREANHNVSAPYGRITLHVFWELNYDFLPNYCYNGSTNRFVRTVLPFSQEFQRDKQPNAQPQYLHGSKALNLAYSSIYGSYRNFVGPPHFQVICRLLGYQGIAVVMEELLKVVKSLLQGTILQYVKTLMEVMPKICRLPRHEYGSPGILEFFHHQLKDIVEYAELKTVCFQNLREVGNAILFCLLIEQSLSLEEVCDLLHAAPFQNILPRVHVKEGERLDAKMKRLESKYAPLHLVPLIERLGTPQQIAIAREGDLLTKERLCCGLSMFEVILTRIRSFLDDPIWRGPLPSNGVMHVDECVEFHRLWSAMQFVYCIPVGTHEFTVEQCFGDGLHWAGCMIIVLLGQQRRFAVLDFCYHLLKVQKHDGKDEIIKNVPLKKMVERIRKFQILNDEIITILDKYLKSGDGEGTPVEHVRCFQPPIHQSLASS",
# FA2H_HUMAN
"MAPAPPPAASFSPSEVQRRLAAGACWVRRGARLYDLSSFVRHHPGGEQLLRARAGQDISADLDGPPHRHSANARRWLEQYYVGELRGEQQGSMENEPVALEETQKTDPAMEPRFKVVDWDKDLVDWRKPLLWQVGHLGEKYDEWVHQPVTRPIRLFHSDLIEGLSKTVWYSVPIIWVPLVLYLSWSYYRTFAQGNVRLFTSFTTEYTVAVPKSMFPGLFMLGTFLWSLIEYLIHRFLFHMKPPSDSYYLIMLHFVMHGQHHKAPFDGSRLVFPPVPASLVIGVFYLCMQLILPEAVGGTVFAGGLLGYVLYDMTHYYLHFGSPHKGSYLYSLKAHHVKHHFAHQKSGFGISTKLWDYCFHTLTPEKPHLKTQ",
# LIGO2_HUMAN
"MLHTAISCWQPFLGLAVVLIFMGSTIGCPARCECSAQNKSVSCHRRRLIAIPEGIPIETKILDLSKNRLKSVNPEEFISYPLLEEIDLSDNIIANVEPGAFNNLFNLRSLRLKGNRLKLVPLGVFTGLSNLTKLDISENKIVILLDYMFQDLHNLKSLEVGDNDLVYISHRAFSGLLSLEQLTLEKCNLTAVPTEALSHLRSLISLHLKHLNINNMPVYAFKRLFHLKHLEIDYWPLLDMMPANSLYGLNLTSLSVTNTNLSTVPFLAFKHLVYLTHLNLSYNPISTIEAGMFSDLIRLQELHIVGAQLRTIEPHSFQGLRFLRVLNVSQNLLETLEENVFSSPRALEVLSINNNPLACDCRLLWILQRQPTLQFGGQQPMCAGPDTIRERSFKDFHSTALSFYFTCKKPKIREKKLQHLLVDEGQTVQLECSADGDPQPVISWVTPRRRFITTKSNGRATVLGDGTLEIRFAQDQDSGMYVCIASNAAGNDTFTASLTVKGFASDRFLYANRTPMYMTDSNDTISNGTNANTFSLDLKTILVSTAMGCFTFLGVVLFCFLLLFVWSRGKGKHKNSIDLEYVPRKNNGAVVEGEVAGPRRFNMKMI",
# ARC_HUMAN
"MELDHRTSGGLHAYPGPRGGQVAKPNVILQIGKCRAEMLEHVRRTHRHLLAEVSKQVERELKGLHRSVGKLESNLDGYVPTSDSQRWKKSIKACLCRCQETIANLERWVKREMHVWREVFYRLERWADRLESTGGKYPVGSESARHTVSVGVGGPESYCHEADGYDYTVSPYAITPPPAAGELPGQEPAEAQQYQPWVPGEDGQPSPGVDTQIFEDPREFLSHLEEYLRQVGGSEEYWLSQIQNHMNGPAKKWWEFKQGSVKNWVEFKKEFLQYSEGTLSREAIQRELDLPQKQGEPLDQFLWRKRDLYQTLYVDADEEEIIQYVVGTLQPKLKRFLRHPLPKTLEQLIQRGMEVQDDLEQAAEPAGPHLPVEDEAETLTPAPNSESVASDRTQPE",
# PLPR4_HUMAN
"MQRAGSSGGRGECDISGAGRLGLEEAARLSCAVHTSPGGGRRPGQAAGMSAKERPKGKVIKDSVTLLPCFYFVELPILASSVVSLYFLELTDVFKPVHSGFSCYDRSLSMPYIEPTQEAIPFLMLLSLAFAGPAITIMVGEGILYCCLSKRRNGVGLEPNINAGGCNFNSFLRRAVRFVGVHVFGLCSTALITDIIQLSTGYQAPYFLTVCKPNYTSLNVSCKENSYIVEDICSGSDLTVINSGRKSFPSQHATLAAFAAVYVSMYFNSTLTDSSKLLKPLLVFTFIICGIICGLTRITQYKNHPVDVYCGFLIGGGIALYLGLYAVGNFLPSDESMFQHRDALRSLTDLNQDPNRLLSAKNGSSSDGIAHTEGILNRNHRDASSLTNLKRANADVEIITPRSPMGKENMVTFSNTLPRANTPSVEDPVRRNASIHASMDSARSKQLLTQWKNKNESRKLSLQVIEPEPGQSPPRSIEMRSSSEPSRVGVNGDHHGPGNQYLKIQPGAVPGCNNSMPGGPRVSIQSRPGSSQLVHIPEETQENISTSPKSSSARAKWLKAAEKTVACNRSNSQPRIMQVIAMSKQQGVLQSSPKNTEGSTVSCTGSIRYKTLTDHEPSGIVRVEAHPENNRPIIQIPSTEGEGSGSWKWKAPEKGSLRQTYELNDLNRDSESCESLKDSFGSGDRKRSNIDSNEHHHHGITTIRVTPVEGSEIGSETLSISSSRDSTLRRKGNIILIPERSNSPENTRNIFYKGTSPTRAYKD",
# NEGR1_HUMAN
"MDMMLLVQGACCSNQWLAAVLLSLCCLLPSCLPAGQSVDFPWAAVDNMMVRKGDTAVLRCYLEDGASKGAWLNRSSIIFAGGDKWSVDPRVSISTLNKRDYSLQIQNVDVTDDGPYTCSVQTQHTPRTMQVHLTVQVPPKIYDISNDMTVNEGTNVTLTCLATGKPEPSISWRHISPSAKPFENGQYLDIYGITRDQAGEYECSAENDVSFPDVRKVKVVVNFAPTIQEIKSGTVTPGRSGLIRCEGAGVPPPAFEWYKGEKKLFNGQQGIIIQNFSTRSILTVTNVTQEHFGNYTCVAANKLGTTNASLPLNPPSTAQYGITGSADVLFSCWYLVLTLSSFTSIFYLKNAILQ",
# ATG9A_HUMAN
"MAQFDTEYQRLEASYSDSPPGEEDLLVHVAEGSKSPWHHIENLDLFFSRVYNLHQKNGFTCMLIGEIFELMQFLFVVAFTTFLVSCVDYDILFANKMVNHSLHPTEPVKVTLPDAFLPAQVCSARIQENGSLITILVIAGVFWIHRLIKFIYNICCYWEIHSFYLHALRIPMSALPYCTWQEVQARIVQTQKEHQICIHKRELTELDIYHRILRFQNYMVALVNKSLLPLRFRLPGLGEAVFFTRGLKYNFELILFWGPGSLFLNEWSLKAEYKRGGQRLELAQRLSNRILWIGIANFLLCPLILIWQILYAFFSYAEVLKREPGALGARCWSLYGRCYLRHFNELEHELQSRLNRGYKPASKYMNCFLSPLLTLLAKNGAFFAGSILAVLIALTIYDEDVLAVEHVLTTVTLLGVTVTVCRSFIPDQHMVFCPEQLLRVILAHIHYMPDHWQGNAHRSQTRDEFAQLFQYKAVFILEELLSPIVTPLILIFCLRPRALEIIDFFRNFTVEVVGVGDTCSFAQMDVRQHGHPQWLSAGQTEASVYQQAEDGKTELSLMHFAITNPGWQPPRESTAFLGFLKEQVQRDGAAASLAQGGLLPENALFTSIQSLQSESEPLSLIANVVAGSSCRGPPLPRDLQGSRHRAEVASALRSFSPLQPGQAPTGRAHSTMTGSGVDARTASSGSSVWEGQLQSLVLSEYASTEMSLHALYMHQLHKQQAQAEPERHVWHRRESDESGESAPDEGGEGARAPQSIPRSASYPCAAPRPGAPETTALHGGFQRRYGGITDPGTVPRVPSHFSRLPLGGWAEDGQSASRHPEPVPEEGSEDELPPQVHKV",
# PRIC2_HUMAN
"MVTVMPLEMEKTISKLMFDFQRNSTSDDDSGCALEEYAWVPPGLKPEQVHQYYSCLPEEKVPYVNSPGEKLRIKQLLHQLPPHDNEVRYCNSLDEEEKRELKLFSSQRKRENLGRGNVRPFPVTMTGAICEQCGGQINGGDIAVFASRAGHGVCWHPPCFVCTVCNELLVDLIYFYQDGKIYCGRHHAECLKPRCAACDEIIFADECTEAEGRHWHMKHFCCFECETVLGGQRYIMKEGRPYCCHCFESLYAEYCDTCAQHIGIDQGQMTYDGQHWHATETCFCCAHCKKSLLGRPFLPKQGQIFCSRACSAGEDPNGSDSSDSAFQNARAKESRRSAKIGKNKGKTEEPMLNQHSQLQVSSNRLSADVDPLSLQMDMLSLSSQTPSLNRDPIWRSREEPYHYGNKMEQNQTQSPLQLLSQCNIRTSYSPGGQGAGAQPEMWGKHFSNPKRSSSLAMTGHAGSFIKECREDYYPGRLRSQESYSDMSSQSFSETRGSIQVPKYEEEEEEEGGLSTQQCRTRHPISSLKYTEDMTPTEQTPRGSMESLALSNATGLSADGGAKRQEHLSRFSMPDLSKDSGMNVSEKLSNMGTLNSSMQFRSAESVRSLLSAQQYQEMEGNLHQLSNPIGYRDLQSHGRMHQSFDFDGGMAGSKLPGQEGVRIQPMSERTRRRATSRDDNRRFRPHRSRRSRRSRSDNALHLASEREAISRLKDRPPLRAREDYDQFMRQRSFQESMGHGSRRDLYGQCPRTVSDLALQNAFGDRWGPYFAEYDWCSTCSSSSESDNEGYFLGEPIPQPARLRYVTSDELLHKYSSYGLPKSSTLGGRGQLHSRKRQKSKNCIIS",
# UBE3B_HUMAN
"MFTLSQTSRAWFIDRARQAREERLVQKERERAAVVIQAHVRSFLCRSRLQRDIRREIDDFFKADDPESTKRSALCIFKIARKLLFLFRIKEDNERFEKLCRSILSSMDAENEPKVWYVSLACSKDLTLLWIQQIKNILWYCCDFLKQLKPEILQDSRLITLYLTMLVTFTDTSTWKILRGKGESLRPAMNHICANIMGHLNQHGFYSVLQILLTRGLARPRPCLSKGTLTAAFSLALRPVIAAQFSDNLIRPFLIHIMSVPALVTHLSTVTPERLTVLESHDMLRKFIIFLRDQDRCRDVCESLEGCHTLCLMGNLLHLGSLSPRVLEEETDGFVSLLTQTLCYCRKYVSQKKSNLTHWHPVLGWFSQSVDYGLNESMHLITKQLQFLWGVPLIRIFFCDILSKKLLESQEPAHAQPASPQNVLPVKSLLKRAFQKSASVRNILRPVGGKRVDSAEVQKVCNICVLYQTSLTTLTQIRLQILTGLTYLDDLLPKLWAFICELGPHGGLKLFLECLNNDTEESKQLLAMLMLFCDCSRHLITILDDIEVYEEQISFKLEELVTISSFLNSFVFKMIWDGIVENAKGETLELFQSVHGWLMVLYERDCRRRFTPEDHWLRKDLKPSVLFQELDRDRKRAQLILQYIPHVIPHKNRVLLFRTMVTKEKEKLGLVETSSASPHVTHITIRRSRMLEDGYEQLRQLSQHAMKGVIRVKFVNDLGVDEAGIDQDGVFKEFLEEIIKRVFDPALNLFKTTSGDERLYPSPTSYIHENYLQLFEFVGKMLGKAVYEGIVVDVPFASFFLSQLLGHHHSVFYSSVDELPSLDSEFYKNLTSIKRYDGDITDLGLTLSYDEDVMGQLVCHELIPGGKTIPVTNENKISYIHLMAHFRMHTQIKNQTAALISGFRSIIKPEWIRMFSTPELQRLISGDNAEIDLEDLKKHTVYYGGFHGSHRVIIWLWDILASDFTPDERAMFLKFVTSCSRPPLLGFAYLKPPFSIRCVEVSDDQDTGDTLGSVLRGFFTIRKREPGGRLPTSSTCFNLLKLPNYSKKSVLREKLRYAISMNTGFELS",
# CSMD2_HUMAN
"MRLISIAPGPRWQVQSHHPRSAGQNCTFQLHGPNGTVESPGFPYGYPNYANCTWTITAEEQHRIQLVFQSFALEEDFDVLSVFDGPPQPENLRTRLTGFQLPATIVSAATTLSLRLISDYAVSAQGFHATYEVLPSHTCGNPGRLPNGIQQGSTFNLGDKVRYSCNLGFFLEGHAVLTCHAGSENSATWDFPLPSCRADDACGGTLRGQSGIISSPHFPSEYHNNADCTWTILAELGDTIALVFIDFQLEDGYDFLEVTGTEGSSLWFTGASLPAPVISSKNWLRLHFTSDGNHRQRGFSAQYQVKKQIELKSRGVKLMPSKDNSQKTSVLTQVGVSQGHNMCPDPGIPERGKRLGSDFRLGSSVQFTCNEGYDLQGSKRITCMKVSDMFAAWSDHRPVCRARMCDAHLRGPSGIITSPNFPIQYDNNAHCVWIITALNPSKVIKLAFEEFDLERGYDTLTVGDGGQDGDQKTVLYILTGTSVPDLIVSTNHQMWLLFQTDGSGSSLGFKASYEEIEQGSCGDPGIPAYGRREGSRFHHGDTLKFECQPAFELVGQKAITCQKNNQWSAKKPGCVFSCFFNFTSPSGVVLSPNYPEDYGNHLHCVWLILARPESRIHLAFNDIDVEPQFDFLVIKDGATAEAPVLGTFSGNQLPSSITSSGHVARLEFQTDHSTGKRGFNITFTTFRHNECPDPGVPVNGKRFGDSLQLGSSISFLCDEGFLGTQGSETITCVLKEGSVVWNSAVLRCEAPCGGHLTSPSGTILSPGWPGFYKDALSCAWVIEAQPGYPIKITFDRFKTEVNYDTLEVRDGRTYSAPLIGVYHGTQVPQFLISTSNYLYLLFSTDKSHSDIGFQLRYETITLQSDHCLDPGIPVNGQRHGNDFYVGALVTFSCDSGYTLSDGEPLECEPNFQWSRALPSCEALCGGFIQGSSGTILSPGFPDFYPNNLNCTWIIETSHGKGVFFTFHTFHLESGHDYLLITENGSFTQPLRQLTGSRLPAPISAGLYGNFTAQVRFISDFSMSYEGFNITFSEYDLEPCEEPEVPAYSIRKGLQFGVGDTLTFSCFPGYRLEGTARITCLGGRRRLWSSPLPRCVAECGNSVTGTQGTLLSPNFPVNYNNNHECIYSIQTQPGKGIQLKARAFELSEGDVLKVYDGNNNSARLLGVFSHSEMMGVTLNSTSSSLWLDFITDAENTSKGFELHFSSFELIKCEDPGTPKFGYKVHDEGHFAGSSVSFSCDPGYSLRGSEELLCLSGERRTWDRPLPTCVAECGGTVRGEVSGQVLSPGYPAPYEHNLNCIWTIEAEAGCTIGLHFLVFDTEEVHDVLRIWDGPVESGVLLKELSGPALPKDLHSTFNSVVLQFSTDFFTSKQGFAIQFSVSTATSCNDPGIPQNGSRSGDSWEAGDSTVFQCDPGYALQGSAEISCVKIENRFFWQPSPPTCIAPCGGDLTGPSGVILSPNYPEPYPPGKECDWKVTVSPDYVIALVFNIFNLEPGYDFLHIYDGRDSLSPLIGSFYGSQLPGRIESSSNSLFLAFRSDASVSNAGFVIDYTENPRESCFDPGSIKNGTRVGSDLKLGSSVTYYCHGGYEVEGTSTLSCILGPDGKPVWNNPRPVCTAPCGGQYVGSDGVVLSPNYPQNYTSGQICLYFVTVPKDYVVFGQFAFFHTALNDVVEVHDGHSQHSRLLSSLSGSHTGESLPLATSNQVLIKFSAKGLAPARGFHFVYQAVPRTSATQCSSVPEPRYGKRLGSDFSVGAIVRFECNSGYALQGSPEIECLPVPGALAQWNVSAPTCVVPCGGNLTERRGTILSPGFPEPYLNSLNCVWKIVVPEGAGIQIQVVSFVTEQNWDSLEVFDGADNTVTMLGSFSGTTVPALLNSTSNQLYLHFYSDISVSAAGFHLEYKTVGLSSCPEPAVPSNGVKTGERYLVNDVVSFQCEPGYALQGHAHISCMPGTVRRWNYPPPLCIAQCGGTVEEMEGVILSPGFPGNYPSNMDCSWKIALPVGFGAHIQFLNFSTEPNHDYIEIRNGPYETSRMMGRFSGSELPSSLLSTSHETTVYFHSDHSQNRPGFKLEYQDLTYSHQISSFLRGFDLSELERTNSTPPVAASYVWDLDPGCEAYELQECPDPEPFANGIVRGAGYNVGQSVTFECLPGYQLTGHPVLTCQHGTNRNWDHPLPKCEVPCGGNITSSNGTVYSPGFPSPYSSSQDCVWLITVPIGHGVRLNLSLLQTEPSGDFITIWDGPQQTAPRLGVFTRSMAKKTVQSSSNQVLLKFHRDAATGGIFAIAFSAYPLTKCPPPTILPNAEVVTENEEFNIGDIVRYRCLPGFTLVGNEILTCKLGTYLQFEGPPPICEVHCPTNELLTDSTGVILSQSYPGSYPQFQTCSWLVRVEPDYNISLTVEYFLSEKQYDEFEIFDGPSGQSPLLKALSGNYSAPLIVTSSSNSVYLRWSSDHAYNRKGFKIRYSAPYCSLPRAPLHGFILGQTSTQPGGSIHFGCNAGYRLVGHSMAICTRHPQGYHLWSEAIPLCQALSCGLPEAPKNGMVFGKEYTVGTKAMYSCSEGYHLQAGAEATAECLDTGLWSNRNVPPQCVPVTCPDVSSISVEHGRWRLIFETQYQFQAQLMLICDPGYYYTGQRVIRCQANGKWSLGDSTPTCRIISCGELPIPPNGHRIGTLSVYGATAIFSCNSGYTLVGSRVRECMANGLWSGSEVRCLAGHCGTPEPIVNGHINGENYSYRGSVVYQCNAGFRLIGMSVRICQQDHHWSGKTPFCVLVSCGHPGSPPHSQMSGDSYTVGAVVRYSCIGKRTLVGNSTRMCGLDGHWTGSLPHCSGTSVGVCGDPGIPAHGIRLGDSFDPGTVMRFSCEAGHVLRGSSERTCQANGSWSGSQPECGVISCGNPGTPSNARVVFSDGLVFSSSIVYECREGYYATGLLSRHCSVNGTWTGSDPECLVINCGDPGIPANGLRLGNDFRYNKTVTYQCVPGYMMESHRVSVLSCTKDRTWNGTKPVCKALMCKPPPLIPNGKVVGSDFMWGSSVTYACLEGYQLSLPAVFTCEGNGSWTGELPQCFPVFCGDPGVPSRGRREDRGFSYRSSVSFSCHPPLVLVGSPRRFCQSDGTWSGTQPSCIDPTLTTCADPGVPQFGIQNNSQGYQVGSTVLFRCQKGYLLQGSTTRTCLPNLTWSGTPPDCVPHHCRQPETPTHANVGALDLPSMGYTLIYSCQEGFSLKGGSEHRTCKADGSWTGKPPICLEVRPSGRPINTAREPPLTQALIPGDVFAKNSLWKGAYEYQGKKQPAMLRVTGFQVANSKVNATMIDHSGVELHLAGTYKKEDFHLLLQVYQITGPVEIFMNKFKDDHWALDGHVSSESSGATFIYQGSVKGQGFGQFGFQRLDLRLLESDPESIGRHFASNSSSVAAAILVPFIALIIAGFVLYLYKHRRRPKVPFNGYAGHENTNVRATFENPMYDRNIQPTDIMASEAEFTVSTVCTAV",
# KI21A_HUMAN
"MLGAPDESSVRVAVRIRPQLAKEKIEGCHICTSVTPGEPQVFLGKDKAFTFDYVFDIDSQQEQIYIQCIEKLIEGCFEGYNATVFAYGQTGAGKTYTMGTGFDVNIVEEELGIISRAVKHLFKSIEEKKHIAIKNGLPAPDFKVNAQFLELYNEEVLDLFDTTRDIDAKSKKSNIRIHEDSTGGIYTVGVTTRTVNTESEMMQCLKLGALSRTTASTQMNVQSSRSHAIFTIHVCQTRVCPQIDADNATDNKIISESAQMNEFETLTAKFHFVDLAGSERLKRTGATGERAKEGISINCGLLALGNVISALGDKSKRATHVPYRDSKLTRLLQDSLGGNSQTIMIACVSPSDRDFMETLNTLKYANRARNIKNKVMVNQDRASQQINALRSEITRLQMELMEYKTGKRIIDEEGVESINDMFHENAMLQTENNNLRVRIKAMQETVDALRSRITQLVSDQANHVLARAGEGNEEISNMIHSYIKEIEDLRAKLLESEAVNENLRKNLTRATARAPYFSGSSTFSPTILSSDKETIEIIDLAKKDLEKLKRKEKRKKKRLQKLEESNREERSVAGKEDNTDTDQEKKEEKGVSERENNELEVEESQEVSDHEDEEEEEEEEEDDIDGGESSDESDSESDEKANYQADLANITCEIAIKQKLIDELENSQKRLQTLKKQYEEKLMMLQHKIRDTQLERDQVLQNLGSVESYSEEKAKKVRSEYEKKLQAMNKELQRLQAAQKEHARLLKNQSQYEKQLKKLQQDVMEMKKTKVRLMKQMKEEQEKARLTESRRNREIAQLKKDQRKRDHQLRLLEAQKRNQEVVLRRKTEEVTALRRQVRPMSDKVAGKVTRKLSSSDAPAQDTGSSAAAVETDASRTGAQQKMRIPVARVQALPTPATNGNRKKYQRKGLTGRVFISKTARMKWQLLERRVTDIIMQKMTISNMEADMNRLLKQREELTKRREKLSKRREKIVKENGEGDKNVANINEEMESLTANIDYINDSISDCQANIMQMEEAKEEGETLDVTAVINACTLTEARYLLDHFLSMGINKGLQAAQKEAQIKVLEGRLKQTEITSATQNQLLFHMLKEKAELNPELDALLGHALQDLDSVPLENVEDSTDEDAPLNSPGSEGSTLSSDLMKLCGEVKPKNKARRRTTTQMELLYADSSELASDTSTGDASLPGPLTPVAEGQEIGMNTETSGTSAREKELSPPPGLPSKIGSISRQSSLSEKKIPEPSPVTRRKAYEKAEKSKAKEQKHSDSGTSEASLSPPSSPPSRPRNELNVFNRLTVSQGNTSVQQDKSDESDSSLSEVHRSSRRGIINPFPASKGIRAFPLQCIHIAEGHTKAVLCVDSTDDLLFTGSKDRTCKVWNLVTGQEIMSLGGHPNNVVSVKYCNYTSLVFTVSTSYIKVWDIRDSAKCIRTLTSSGQVTLGDACSASTSRTVAIPSGENQINQIALNPTGTFLYAASGNAVRMWDLKRFQSTGKLTGHLGPVMCLTVDQISSGQDLIITGSKDHYIKMFDVTEGALGTVSPTHNFEPPHYDGIEALTIQGDNLFSGSRDNGIKKWDLTQKDLLQQVPNAHKDWVCALGVVPDHPVLLSGCRGGILKVWNMDTFMPVGEMKGHDSPINAICVNSTHIFTAADDRTVRIWKARNLQDGQISDTGDLGEDIASN",
# MDGA2_HUMAN
"MDLLYGLVWLLTVLLEGISGQGVYAPPTVRIVHSGLACNIEEERYSERVYTIREGETLELTCLVTGHPRPQIRWTKTAGSASDRFQDSSVFNETLRITNIQRHQGGRYYCKAENGLGSPAIKSIRVDVYYLDDPVVTVHQSIGEAKEQFYYERTVFLRCVANSNPPVRYSWRRGQEVLLQGSDKGVEIYEPFFTQGETKILKLKNLRPQDYANYSCIASVRNVCNIPDKMVSFRLSNKTASPSIKLLVDDPIVVNPGEAITLVCVTTGGEPAPSLTWVRSFGTLPEKTVLNGGTLTIPAITSDDAGTYSCIANNNVGNPAKKSTNIIVRALKKGRFWITPDPYHKDDNIQIGREVKISCQVEAVPSEELTFSWFKNGRPLRSSERMVITQTDPDVSPGTTNLDIIDLKFTDFGTYTCVASLKGGGISDISIDVNISSSTVPPNLTVPQEKSPLVTREGDTIELQCQVTGKPKPIILWSRADKEVAMPDGSMQMESYDGTLRIVNVSREMSGMYRCQTSQYNGFNVKPREALVQLIVQYPPAVEPAFLEIRQGQDRSVTMSCRVLRAYPIRVLTYEWRLGNKLLRTGQFDSQEYTEYAVKSLSNENYGVYNCSIINEAGAGRCSFLVTGKAYAPEFYYDTYNPVWQNRHRVYSYSLQWTQMNPDAVDRIVAYRLGIRQAGQQRWWEQEIKINGNIQKGELITYNLTELIKPEAYEVRLTPLTKFGEGDSTIRVIKYSAPVNPHLREFHCGFEDGNICLFTQDDTDNFDWTKQSTATRNTKYTPNTGPNADRSGSKEGFYMYIETSRPRLEGEKARLLSPVFSIAPKNPYGPTNTAYCFSFFYHMYGQHIGVLNVYLRLKGQTTIENPLWSSSGNKGQRWNEAHVNIYPITSFQLIFEGIRGPGIEGDIAIDDVSIAEGECAKQDLATKNSVDGAVGILVHIWLFPIIVLISILSPRR",
# RHG22_HUMAN
"MLSPKIRQARRARSKSLVMGEQSRSPGRMPCPHRLGPVLKAGWLKKQRSIMKNWQQRWFVLRGDQLFYYKDKDEIKPQGFISLQGTQVTELPPGPEDPGKHLFEISPGGAGEREKVPANPEALLLMASSQRDMEDWVQAIRRVIWAPLGGGIFGQRLEETVHHERKYGPRLAPLLVEQCVDFIRERGLTEEGLFRMPGQANLVRDLQDSFDCGEKPLFDSTTDVHTVASLLKLYLRELPEPVVPFARYEDFLSCAQLLTKDEGEGTLELAKQVSNLPQANYNLLRYICKFLDEVQAYSNVNKMSVQNLATVFGPNILRPQVEDPVTIMEGTSLVQHLMTVLIRKHSQLFTAPVPEGPTSPRGGLQCAVGWGSEEVTRDSQGEPGGPGLPAHRTSSLDGAAVAVLSRTAPTGPGSRCSPGKKVQTLPSWKSSFRQPRSLSGSPKGGGSSLEVPIISSGGNWLMNGLSSLRGHRRASSGDRLKDSGSVQRLSTYDNVPAPGLVPGIPSVASMAWSGASSSESSVGGSLSSCTACRASDSSARSSLHTDWALEPSPLPSSSEDPKSLDLDHSMDEAGAGASNSEPSEPDSPTREHARRSEALQGLVTELRAELCRQRTEYERSVKRIEEGSADLRKRMSRLEEELDQEKKKYIMLEIKLRNSERAREDAERRNQLLQREMEEFFSTLGSLTVGAKGARAPK",
# C1QL2_HUMAN
"MALGLLIAVPLLLQAAPRGAAHYEMMGTCRMICDPYTAAPGGEPPGAKAQPPGPSTAALEVMQDLSANPPPPFIQGPKGDPGRPGKPGPRGPPGEPGPPGPRGPPGEKGDSGRPGLPGLQLTAGTASGVGVVGGGAGVGGDSEGEVTSALSATFSGPKIAFYVGLKSPHEGYEVLKFDDVVTNLGNHYDPTTGKFSCQVRGIYFFTYHILMRGGDGTSMWADLCKNGQVRASAIAQDADQNYDYASNSVVLHLDSGDEVYVKLDGGKAHGGNNNKYSTFSGFLLYPD",
# CPEB2_HUMAN
"MPPPSPDSENGFYPGLPSSMNPAFFPSFSPVSPHGCTGLSVPTSGGGGGGFGGPFSATAVPPPPPPAMNIPQQQPPPPAAPQQPQSRRSPVSPQLQQQHQAAAAAFLQQRNSYNHHQPLLKQSPWSNHQSSGWGTGSMSWGAMHGRDHRRTGNMGIPGTMNQISPLKKPFSGNVIAPPKFTRSTPSLTPKSWIEDNVFRTDNNSNTLLPLQVRSSLQLPAWGSDSLQDSWCTAAGTSRIDQDRSRMYDSLNMHSLENSLIDIMRAEHDPLKGRLSYPHPGTDNLLMLNGRSSLFPIDDGLLDDGHSDQVGVLNSPTCYSAHQNGERIERFSRKVFVGGLPPDIDEDEITASFRRFGPLVVDWPHKAESKSYFPPKGYAFLLFQEESSVQALIDACIEEDGKLYLCVSSPTIKDKPVQIRPWNLSDSDFVMDGSQPLDPRKTIFVGGVPRPLRAVELAMIMDRLYGGVCYAGIDTDPELKYPKGAGRVAFSNQQSYIAAISARFVQLQHGDIDKRVEVKPYVLDDQMCDECQGARCGGKFAPFFCANVTCLQYYCEFCWANIHSRAGREFHKPLVKEGADRPRQIHFRWN",
# NECA2_HUMAN
"MCERAARLCRAGAHRLLREPPQQGRALGGLLRWVGARMGEPRESLAPAAPADPGPASPRGGTAVILDIFRRADKNDDGKLSLEEFQLFFADGVLNEKELEDLFHTIDSDNTNHVDTKELCDYFVDHMGDYEDVLASLETLNHSVLKAMGYTKKVYEGGSNVDQFVTRFLLKETANQIQSLLSSVESAVEAIEEQTSQLRQNHIKPSHSAAQTWCGSPTPASAPNHKLMAMEQGKTLPSATEDAKEEGLEAQISRLAELIGRLESKALWFDLQQRLSDEDGTNMHLQLVRQEMAVCPEQLSEFLDSLRQYLRGTTGVRNCFHITAVRLSDGFTFVIYEFWETEEAWKRHLQSPLCKAFRHVKVDTLSQPEALSRILVPAAWCTVGRD",
# ANS1B_HUMAN
"MGKDQELLEAARTGNVALVEKLLSGRKGGILGGGSGPLPLSNLLSIWRGPNVNCTDSSGYTALHHAALNGHKDIVLKLLQYEASTNVADNKGYFPIHLAAWKGDVEIVKILIHHGPSHSRVNEQNNENETALHCAAQYGHSEVVAVLLEELTDPTIRNSKLETPLDLAALYGRLRVVKMIISAHPNLMSCNTRKHTPLHLAARNGHKAVVQVLLEAGMDVSCQTEKGSALHEAALFGKVDVVRVLLETGIDANIKDSLGRTVLDILKEHPSQKSLQIATLLQEYLEGVGRSTVLEEPVQEDATQETHISSPVESPSQKTKSETVTGELSKLLDEIKLCQEKDYSFEDLCHTISDHYLDNLSKISEEELGKNGSQSVRTSSTINLSPGEVEEEDDDENTCGPSGLWEALTPCNGCRNLGFPMLAQESYPKKRNYTMEIVPSASLDTFPSENENFLCDLMDTAVTKKPCSLEIARAPSPRTDNASEVAVTTPGTSNHRNSSTGPTPDCSPPSPDTALKNIVKVIRPQPKQRTSIVSSLDFHRMNHNQEYFEINTSTGCTSFTASPPASPPTSSVGTTEVKNEGTNHTDDLSRQDDNDPPKEYDPGQFAGLLHGSSPACESPENPFHLYGKREQCEKGQDEVSLANSPLPFKQSPIENNSEPLVKKIKPKVVSRTIFHKKSNQLENHTIVGTRSTRSGSRNGDQWVMNAGGFVERACTLGRIRSLPKALIDMHLSKSVSKSDSDLIAYPSNEKTSRVNWSESSTAEHSSKGNSERTPSFTSEWEEIDKIMSSIDVGINNELKEMNGETTRPRCPVQTVGQWLESIGLPQYENHLMANGFDNVQFMGSNVMEDQDLLEIGILNSGHRQRILQAIQLLPKMRPIGHDGYHPTSVAEWLDSIELGDYTKAFLINGYTSMDLLKKIWEVELINVLKINLIGHRKRILASLGDRLHDDPPQKPPRSITLREPSGNHTPPQLSPSLSQSTYTTGGSLDVPHIIMQGDARRRRNENYFDDIPRSKLERQMAQTGDWGEPSITLRPPNEATASTPVQYWQHHPEKLIFQSCDYKAFYLGSMLIKELRGTESTQDACAKMRANCQKSTEQMKKVPTIILSVSYKGVKFIDATNKNIIAEHEIRNISCAAQDPEDLSTFAYITKDLKSNHHYCHVFTAFDVNLAYEIILTLGQAFEVAYQLALQARKGGHSSTLPESFENKPSKPIPKPRVSIRKSVDLLHASHTGQEPSERHTEEALRKF",
# GRASP_HUMAN
"MTLRRLRKLQQKEEAAATPDPAARTPDSEVAPAAPVPTPGPPAAAATPGPPADELYAALEDYHPAELYRALAVSGGTLPRRKGSGFRWKNLSQSPEQQRKVLTLEKEDNQTFGFEIQTYGLHHREEQRVEMVTFVCRVHESSPAQLAGLTPGDTIASVNGLNVEGIRHREIVDIIKASGNVLRLETLYGTSIRKAELEARLQYLKQTLYEKWGEYRSLMVQEQRLVHGLVVKDPSIYDTLESVRSCLYGAGLLPGSLPFGPLLAVPGRPRGGARRARGDADDAVYHTCFFGDSEPPALPPPPPPARAFGPGPAETPAVGPGPGPRAALSRSASVRCAGPGGGGGGGAPGALWTEAREQALCGPGLRKTKYRSFRRRLLKFIPGLNRSLEEEESQL",
# PRRT2_HUMAN
"MAASSSEISEMKGVEESPKVPGEGPGHSEAETGPPQVLAGVPDQPEAPQPGPNTTAAPVDSGPKAGLAPETTETPAGASETAQATDLSLSPGGESKANCSPEDPCQETVSKPEVSKEATADQGSRLESAAPPEPAPEPAPQPDPRPDSQPTPKPALQPELPTQEDPTPEILSESVGEKQENGAVVPLQAGDGEEGPAPEPHSPPSKKSPPANGAPPRVLQQLVEEDRMRRAHSGHPGSPRGSLSRHPSSQLAGPGVEGGEGTQKPRDYIILAILSCFCPMWPVNIVAFAYAVMSRNSLQQGDVDGAQRLGRVAKLLSIVALVGGVLIIIASCVINLGVYK",
# FLIP1_HUMAN
"MRSRNQGGESASDGHISCPKPSIIGNAGEKSLSEDAKKKKKSNRKEDDVMASGTVKRHLKTSGECERKTKKSLELSKEDLIQLLSIMEGELQAREDVIHMLKTEKTKPEVLEAHYGSAEPEKVLRVLHRDAILAQEKSIGEDVYEKPISELDRLEEKQKETYRRMLEQLLLAEKCHRRTVYELENEKHKHTDYMNKSDDFTNLLEQERERLKKLLEQEKAYQARKEKENAKRLNKLRDELVKLKSFALMLVDERQMHIEQLGLQSQKVQDLTQKLREEEEKLKAITSKSKEDRQKLLKLEVDFEHKASRFSQEHEEMNAKLANQESHNRQLRLKLVGLTQRIEELEETNKNLQKAEEELQELRDKIAKGECGNSSLMAEVENLRKRVLEMEGKDEEITKTESQCRELRKKLQEEEHHSKELRLEVEKLQKRMSELEKLEEAFSKSKSECTQLHLNLEKEKNLTKDLLNELEVVKSRVKELECSESRLEKAELSLKDDLTKLKSFTVMLVDERKNMMEKIKQEERKVDGLNKNFKVEQGKVMDVTEKLIEESKKLLKLKSEMEEKVYNLTRERDELIGKLKSEEEKSSELSCSVDLLKKRLDGIEEVEREITRGRSRKGSELTCPEDNKIKELTLEIERLKKRLQQLEVVEGDLMKTEDEYDQLEQKFRTEQDKANFLSQQLEEIKHQIAKNKAIEKGEVVSQEAELRHRFRLEEAKSRDLKAEVQALKEKIHELMNKEDQLSQLQVDYSVLQQRFMEEENKNKNMGQEVLNLTKELELSKRYSRALRPSVNGRRMVDVPVTSTGVQTDAVSGEAAEEETPAVFIRKSFQEENHIMSNLRQVGLKKPVERSSVLDRYPPAANELTMRKSWIPWMRKRENGPSITQEKGPRTNSSPGHPGEVVLSPKQGQPLHIRVTPDHENSTATLEITSPTSEEFFSSTTVIPTLGNQKPRITIIPSPNVMPQKQKSGDTTLGPERAMSPVTITTFSREKTPESGRGAFADRPTSPIQIMTVSTSAAPAEIAVSPESQEMPMGRTILKVTPEKQTVPTPVRKYNSNANIITTEDNKIHIHLGSQFKRSPGTSGEGVSPVITVRPVNVTAEKEVSTGTVLRSPRNHLSSRPGASKVTSTITITPVTTSSARGTQSVSGQDGSSQRPTPTRIPMSKGMKAGKPVVAAPGAGNLTKFEPRAETQSMKIELKKSAASSTTSLGGGKG",
# LHPL4_HUMAN
"MLPSQEASKLYHEHYMRNSRAIGVLWAIFTICFAIINVVVFIQPYWVGDSVSTPKPGYFGLFHYCVGSGLAGRELTCRGSFTDFSTIPSSAFKAAAFFVLLSMVLILGCITCFSLFFFCNTATVYKICAWMQLLAALCLVLGCMIFPDGWDAETIRDMCGAKTGKYSLGDCSVRWAYILAIIGILNALILSFLAFVLGNRQTDLLQEELKPENKDFVGSTVSSVLRPGGDVSGWGVLPCPVAHSQGP",
# AGRA1_HUMAN
"MDLKTVLSLPRYPGEFLHPVVYACTAVMLLCLLASFVTYIVHQSAIRISRKGRHTLLNFCFHAALTFTVFAGGINRTKYPILCQAVGIVLHYSTLSTMLWIGVTARNIYKQVTKKAPLCLDTDQPPYPRQPLLRFYLVSGGVPFIICGVTAATNIRNYGTEDEDTAYCWMAWEPSLGAFYGPAAIITLVTCVYFLGTYVQLRRHPGRRYELRTQPEEQRRLATPEGGRGIRPGTPPAHDAPGASVLQNEHSFQAQLRAAAFTLFLFTATWAFGALAVSQGHFLDMVFSCLYGAFCVTLGLFVLIHHCAKREDVWQCWWACCPPRKDAHPALDANGAALGRAACLHSPGLGQPRGFAHPPGPCKMTNLQAAQGHASCLSPATPCCAKMHCEPLTADEAHVHLQEEGAFGHDPHLHGCLQGRTKPPYFSRHPAEEPEYAYHIPSSLDGSPRSSRTDSPPSSLDGPAGTHTLACCTQGDPFPMVTQPEGSDGSPALYSCPTQPGREAALGPGHLEMLRRTQSLPFGGPSQNGLPKGKLLEGLPFGTDGTGNIRTGPWKNETTV",
# SYT9_HUMAN
"MPGARDALCHQALQLLAELCARGALEHDSCQDFIYHLRDRARPRLRDPDISVSLLTLVVTACGLALFGVSLFVSWKLCWVPWRERGLPSGSKDNNQEPLNYMDTETNEQENSEDFLDPPTPCPDSSMKISHTSPDIPLSTQTGIQENCAHGVRVQRQVTEPTSSARHNSIRRQLNLSNPDFNIQQLQKQEQLTGIGRIKPELYKQRSLDNDDGRRSNSKACGKLNFILKYDCDLEQLIVKIHKAVNLPAKDFSGTSDPYVKIYLLPDRKTKHQTKVHRKTLNPVFDEVFLFPVPYNDLEARKLHFSVYDFDRFSRHDLIGQVVVDHFLDLADFPRECILWKDIEYVTNDNVDLGELMFSLCYLPTAGRLTITIIKARNLKAMDITGASDPYVKVSLMCDGRRLKKRKTSTKRNTLNPVYNEAIVFDVPPENIDQIHLSIAVMDYDRVGHNEIIGVCQVGNEAERLGRDHWSEMLSYPRKPIAHWHSLVEKR",
# LRRT1_HUMAN
"MDFLLLGLCLYWLLRRPSGVVLCLLGACFQMLPAAPSGCPQLCRCEGRLLYCEALNLTEAPHNLSGLLGLSLRYNSLSELRAGQFTGLMQLTWLYLDHNHICSVQGDAFQKLRRVKELTLSSNQITQLPNTTFRPMPNLRSVDLSYNKLQALAPDLFHGLRKLTTLHMRANAIQFVPVRIFQDCRSLKFLDIGYNQLKSLARNSFAGLFKLTELHLEHNDLVKVNFAHFPRLISLHSLCLRRNKVAIVVSSLDWVWNLEKMDLSGNEIEYMEPHVFETVPHLQSLQLDSNRLTYIEPRILNSWKSLTSITLAGNLWDCGRNVCALASWLNNFQGRYDGNLQCASPEYAQGEDVLDAVYAFHLCEDGAEPTSGHLLSAVTNRSDLGPPASSATTLADGGEGQHDGTFEPATVALPGGEHAENAVQIHKVVTGTMALIFSFLIVVLVLYVSWKCFPASLRQLRQCFVTQRRKQKQKQTMHQMAAMSAQEYYVDYKPNHIEGALVIINEYGSCTCHQQPARECEV",
# MAGI2_HUMAN
"MSKSLKKKSHWTSKVHESVIGRNPEGQLGFELKGGAENGQFPYLGEVKPGKVAYESGSKLVSEELLLEVNETPVAGLTIRDVLAVIKHCKDPLRLKCVKQGGIVDKDLRHYLNLRFQKGSVDHELQQIIRDNLYLRTVPCTTRPHKEGEVPGVDYIFITVEDFMELEKSGALLESGTYEDNYYGTPKPPAEPAPLLLNVTDQILPGATPSAEGKRKRNKSVSNMEKASIEPPEEEEEERPVVNGNGVVVTPESSEHEDKSAGASGEMPSQPYPAPVYSQPEELKEQMDDTKPTKPEDNEEPDPLPDNWEMAYTEKGEVYFIDHNTKTTSWLDPRLAKKAKPPEECKENELPYGWEKIDDPIYGTYYVDHINRRTQFENPVLEAKRKLQQHNMPHTELGTKPLQAPGFREKPLFTRDASQLKGTFLSTTLKKSNMGFGFTIIGGDEPDEFLQVKSVIPDGPAAQDGKMETGDVIVYINEVCVLGHTHADVVKLFQSVPIGQSVNLVLCRGYPLPFDPEDPANSMVPPLAIMERPPPVMVNGRHNYETYLEYISRTSQSVPDITDRPPHSLHSMPTDGQLDGTYPPPVHDDNVSMASSGATQAELMTLTIVKGAQGFGFTIADSPTGQRVKQILDIQGCPGLCEGDLIVEINQQNVQNLSHTEVVDILKDCPIGSETSLIIHRGGFFSPWKTPKPIMDRWENQGSPQTSLSAPAIPQNLPFPPALHRSSFPDSTEAFDPRKPDPYELYEKSRAIYESRQQVPPRTSFRMDSSGPDYKELDVHLRRMESGFGFRILGGDEPGQPILIGAVIAMGSADRDGRLHPGDELVYVDGIPVAGKTHRYVIDLMHHAARNGQVNLTVRRKVLCGGEPCPENGRSPGSVSTHHSSPRSDYATYTNSNHAAPSSNASPPEGFASHSLQTSDVVIHRKENEGFGFVIISSLNRPESGSTITVPHKIGRIIDGSPADRCAKLKVGDRILAVNGQSIINMPHADIVKLIKDAGLSVTLRIIPQEELNSPTSAPSSEKQSPMAQQSPLAQQSPLAQPSPATPNSPIAQPAPPQPLQLQGHENSYRSEVKARQDVKPDIRQPPFTDYRQPPLDYRQPPGGDYQQPPPLDYRQPPLLDYRQHSPDTRQYPLSDYRQPQDFDYFTVDMEKGAKGFGFSIRGGREYKMDLYVLRLAEDGPAIRNGRMRVGDQIIEINGESTRDMTHARAIELIKSGGRRVRLLLKRGTGQVPEYDEPAPWSSPAAAAPGLPEVGVSLDDGLAPFSPSHPAPPSDPSHQISPGPTWDIKREHDVRKPKELSACGQKKQRLGEQRERSASPQRAARPRLEEAPGGQGRPEAGRPASEARAPGLAAADAADAARAGGKEAPRAAAGSELCRREGPGAAPAFAGPGGGGSGALEAEGRAGARAGPRPGPRPPGGAPARKAAVAPGPWKVPGSDKLPSVLKPGASAASR",
# LHPL3_HUMAN
"MPGAAAAAAAAAAAMLPAQEAAKLYHTNYVRNSRAIGVLWAIFTICFAIVNVVCFIQPYWIGDGVDTPQAGYFGLFHYCIGNGFSRELTCRGSFTDFSTLPSGAFKAASFFIGLSMMLIIACIICFTLFFFCNTATVYKICAWMQLTSAACLVLGCMIFPDGWDSDEVKRMCGEKTDKYTLGACSVRWAYILAIIGILDALILSFLAFVLGNRQDSLMAEELKAENKVLLSQYSLE",
# RIMS1_HUMAN
"MSSAVGPRGPRPPTVPPPMQELPDLSHLTEEERNIIMAVMDRQKEEEEKEEAMLKCVVRDMAKPAACKTPRNAENQPHQPSPRLHQQFESYKEQVRKIGEEARRYQGEHKDDAPTCGICHKTKFADGCGHLCSYCRTKFCARCGGRVSLRSNNEDKVVMWVCNLCRKQQEILTKSGAWFFGSGPQQTSQDGTLSDTATGAGSEVPREKKARLQERSRSQTPLSTAAASSQDAAPPSAPPDRSKGAEPSQQALGPEQKQASSRSRSEPPRERKKTPGLSEQNGKGALKSERKRVPKTSAQPVEGAVEERERKERRESRRLEKGRSQDYPDTPEKRDEGKAADEEKQRKEEDYQTRYRSDPNLARYPVKPPPEEQQMRMHARVSRARHERRHSDVALPRTEAGAALPEGKAGKRAPAAARASPPDSPRAYSAERTAETRAPGAKQLTNHSPPAPRHGPVPAEAPELKAQEPLRKQSRLDPSSAVLMRKAKREKVETMLRNDSLSSDQSESVRPSPPKPHRSKRGGKKRQMSVSSSEEEGVSTPEYTSCEDVELESESVSEKGDLDYYWLDPATWHSRETSPISSHPVTWQPSKEGDRLIGRVILNKRTTMPKDSGALLGLKVVGGKMTDLGRLGAFITKVKKGSLADVVGHLRAGDEVLEWNGKPLPGATNEEVYNIILESKSEPQVEIIVSRPIGDIPRIPESSHPPLESSSSSFESQKMERPSISVISPTSPGALKDAPQVLPGQLSVKLWYDKVGHQLIVNVLQATDLPARVDGRPRNPYVKMYFLPDRSDKSKRRTKTVKKILEPKWNQTFVYSHVHRRDFRERMLEITVWDQPRVQEEESEFLGEILIELETALLDDEPHWYKLQTHDESSLPLPQPSPFMPRRHIHGESSSKKLQRSQRISDSDISDYEVDDGIGVVPPVGYRSSARESKSTTLTVPEQQRTTHHRSRSVSPHRGNDQGKPRSRLPNVPLQRSLDEIHPTRRSRSPTRHHDASRSPVDHRTRDVDSQYLSEQDSELLMLPRAKRGRSAECLHTTRHLVRHYKTLPPKMPLLQSSSHWNIYSSILPAHTKTKSVTRQDISLHHECFNSTVLRFTDEILVSELQPFLDRARSASTNCLRPDTSLHSPERERGRWSPSLDRRRPPSPRIQIQHASPENDRHSRKSERSSIQKQTRKGTASDAERVLPTCLSRRGHAAPRATDQPVIRGKHPARSRSSEHSSIRTLCSMHHLVPGGSAPPSPLLTRMHRQRSPTQSPPADTSFSSRRGRQLPQVPVRSGSIEQASLVVEERTRQMKMKVHRFKQTTGSGSSQELDREQYSKYNIHKDQYRSCDNVSAKSSDSDVSDVSAISRTSSASRLSSTSFMSEQSERPRGRISSFTPKMQGRRMGTSGRSIMKSTSVSGEMYTLEHNDGSQSDTAVGTVGAGGKKRRSSLSAKVVAIVSRRSRSTSQLSQTESGHKKLKSTIQRSTETGMAAEMRKMVRQPSRESTDGSINSYSSEGNLIFPGVRLGADSQFSDFLDGLGPAQLVGRQTLATPAMGDIQIGMEDKKGQLEVEVIRARSLTQKPGSKSTPAPYVKVYLLENGACIAKKKTRIARKTLDPLYQQSLVFDESPQGKVLQVIVWGDYGRMDHKCFMGVAQILLEELDLSSMVIGWYKLFPPSSLVDPTLTPLTRRASQSSLESSTGPPCIRS",
# CAPS2_HUMAN
"MLDPSSSEEESDEGLEEESRDVLVAAGSSQRAPPAPTREGRRDAPGRAGGGGAARSVSPSPSVLSEGRDEPQRQLDDEQERRIRLQLYVFVVRCIAYPFNAKQPTDMARRQQKLNKQQLQLLKERFQAFLNGETQIVADEAFCNAVRSYYEVFLKSDRVARMVQSGGCSANDFREVFKKNIEKRVRSLPEIDGLSKETVLSSWIAKYDAIYRGEEDLCKQPNRMALSAVSELILSKEQLYEMFQQILGIKKLEHQLLYNACQLDNADEQAAQIRRELDGRLQLADKMAKERKFPKFIAKDMENMYIEELRSSVNLLMANLESLPVSKGGPEFKLQKLKRSQNSAFLDIGDENEIQLSKSDVVLSFTLEIVIMEVQGLKSVAPNRIVYCTMEVEGEKLQTDQAEASRPQWGTQGDFTTTHPRPVVKVKLFTESTGVLALEDKELGRVILYPTSNSSKSAELHRMVVPKNSQDSDLKIKLAVRMDKPAHMKHSGYLYALGQKVWKRWKKRYFVLVQVSQYTFAMCSYREKKSEPQELMQLEGYTVDYTDPHPGLQGGCMFFNAVKEGDTVIFASDDEQDRILWVQAMYRATGQSYKPVPAIQTQKLNPKGGTLHADAQLSGKDADRFQKHGMDEFISANPCKLDHAFLFRILQRQTLDHRLNDSYSCLGWFSPGQVFVLDEYCARYGVRGCHRHLCYLAELMEHSENGAVIDPTLLHYSFAFCASHVHGNRPDGIGTVSVEEKERFEEIKERLSSLLENQISHFRYCFPFGRPEGALKATLSLLERVLMKDIATPIPAEEVKKVVRKCLEKAALINYTRLTEYAKIEETMNQASPARKLEEILHLAELCIEVLQQNEEHHAEGREAFAWWPDLLAEHAEKFWALFTVDMDTALEAQPQDSWDSFPLFQLLNNFLRNDTLLCNGKFHKHLQEIFVPLVVRYVDLMESSIAQSIHRGFEQETWQPVKNIANSLPNVALPKVPSLPLNLPQIPNISTASWMPSLYESTNGSATSEDLFWKLDALQMFVFDLHWPEQEFAHHLEQRLKLMASDMLEACVKRTRTAFELKLQKASKTTDLRIPASVCTMFNVLVDAKKQSTKLCALDGGQEQQYHSKIDDLIDNSVKEIISLLVSKFVSVLEGVLSKLSRYDEGTFFSSILSFTVKAAAKYVDVPKPGMDLADTYIMFVRQNQDILREKVNEEMYIEKLFDQWYSSSMKVICVWLTDRLDLQLHIYQLKTLIKIVKKTYRDFRLQGVLEGTLNSKTYDTVHRRLTVEEATASVSEGGGLQGITMKDSDEEEEG",
# LRRT4_HUMAN
"MGFHLITQLKGMSVVLVLLPTLLLVMLTGAQRACPKNCRCDGKIVYCESHAFADIPENISGGSQGLSLRFNSIQKLKSNQFAGLNQLIWLYLDHNYISSVDEDAFQGIRRLKELILSSNKITYLHNKTFHPVPNLRNLDLSYNKLQTLQSEQFKGLRKLIILHLRSNSLKTVPIRVFQDCRNLDFLDLGYNRLRSLSRNAFAGLLKLKELHLEHNQFSKINFAHFPRLFNLRSIYLQWNRIRSISQGLTWTWSSLHNLDLSGNDIQGIEPGTFKCLPNLQKLNLDSNKLTNISQETVNAWISLISITLSGNMWECSRSICPLFYWLKNFKGNKESTMICAGPKHIQGEKVSDAVETYNICSEVQVVNTERSHLVPQTPQKPLIIPRPTIFKPDVTQSTFETPSPSPGFQIPGAEQEYEHVSFHKIIAGSVALFLSVAMILLVIYVSWKRYPASMKQLQQHSLMKRRRKKARESERQMNSPLQEYYVDYKPTNSETMDISVNGSGPCTYTISGSRECEMPHHMKPLPYYSYDQPVIGYCQAHQPLHVTKGYETVSPEQDESPGLELGRDHSFIATIARSAAPAIYLERIAN",
# LRRT3_HUMAN
"MGFNVIRLLSGSAVALVIAPTVLLTMLSSAERGCPKGCRCEGKMVYCESQKLQEIPSSISAGCLGLSLRYNSLQKLKYNQFKGLNQLTWLYLDHNHISNIDENAFNGIRRLKELILSSNRISYFLNNTFRPVTNLRNLDLSYNQLHSLGSEQFRGLRKLLSLHLRSNSLRTIPVRIFQDCRNLELLDLGYNRIRSLARNVFAGMIRLKELHLEHNQFSKLNLALFPRLVSLQNLYLQWNKISVIGQTMSWTWSSLQRLDLSGNEIEAFSGPSVFQCVPNLQRLNLDSNKLTFIGQEILDSWISLNDISLAGNIWECSRNICSLVNWLKSFKGLRENTIICASPKELQGVNVIDAVKNYSICGKSTTERFDLARALPKPTFKPKLPRPKHESKPPLPPTVGATEPGPETDADAEHISFHKIIAGSVALFLSVLVILLVIYVSWKRYPASMKQLQQRSLMRRHRKKKRQSLKQMTPSTQEFYVDYKPTNTETSEMLLNGTGPCTYNKSGSRECEIPLSMNVSTFLAYDQPTISYCGVHHELLSHKSFETNAQEDTMETHLETELDLSTITTAGRISDHKQQLA",
# KCMB4_HUMAN
"MAKLRVAYEYTEAEDKSIRLGLFLIISGVVSLFIFGFCWLSPALQDLQATEANCTVLSVQQIGEVFECTFTCGADCRGTSQYPCVQVYVNNSESNSRALLHSDEHQLLTNPKCSYIPPCKRENQKNLESVMNWQQYWKDEIGSQPFTCYFNQHQRPDDVLLHRTHDEIVLLHCFLWPLVTFVVGVLIVVLTICAKSLAVKAEAMKKRKFS",
# STX12_HUMAN
"MSYGPLDMYRNPGPSGPQLRDFSSIIQTCSGNIQRISQATAQIKNLMSQLGTKQDSSKLQENLQQLQHSTNQLAKETNELLKELGSLPLPLSTSEQRQQRLQKERLMNDFSAALNNFQAVQRRVSEKEKESIARARAGSRLSAEERQREEQLVSFDSHEEWNQMQSQEDEVAITEQDLELIKERETAIRQLEADILDVNQIFKDLAMMIHDQGDLIDSIEANVESSEVHVERATEQLQRAAYYQKKSRKKMCILVLVLSVIILILGLIIWLVYKTK",
# HOME1_HUMAN
"MGEQPIFSTRAHVFQIDPNTKKNWVPTSKHAVTVSYFYDSTRNVYRIISLDGSKAIINSTITPNMTFTKTSQKFGQWADSRANTVYGLGFSSEHHLSKFAEKFQEFKEAARLAKEKSQEKMELTSTPSQESAGGDLQSPLTPESINGTDDERTPDVTQNSEPRAEPTQNALPFSHSSAISKHWEAELATLKGNNAKLTAALLESTANVKQWKQQLAAYQEEAERLHKRVTELECVSSQANAVHTHKTELNQTIQELEETLKLKEEEIERLKQEIDNARELQEQRDSLTQKLQEVEIRNKDLEGQLSDLEQRLEKSQNEQEAFRNNLKTLLEILDGKIFELTELRDNLAKLLECS",
# MIB1_HUMAN
"MSNSRNNRVMVEGVGARVVRGPDWKWGKQDGGEGHVGTVRSFESPEEVVVVWDNGTAANYRCSGAYDLRILDSAPTGIKHDGTMCDTCRQQPIIGIRWKCAECTNYDLCTVCYHGDKHHLRHRFYRITTPGSERVLLESRRKSKKITARGIFAGARVVRGVDWQWEDQDGGNGRRGKVTEIQDWSASSPHSAAYVLWDNGAKNLYRVGFEGMSDLKCVQDAKGGSFYRDHCPVLGEQNGNRNPGGLQIGDLVNIDLDLEIVQSLQHGHGGWTDGMFETLTTTGTVCGIDEDHDIVVQYPSGNRWTFNPAVLTKANIVRSGDAAQGAEGGTSQFQVGDLVQVCYDLERIKLLQRGHGEWAEAMLPTLGKVGRVQQIYSDSDLKVEVCGTSWTYNPAAVSKVASAGSAISNASGERLSQLLKKLFETQESGDLNEELVKAAANGDVAKVEDLLKRPDVDVNGQCAGHTAMQAASQNGHVDILKLLLKQNVDVEAEDKDGDRAVHHAAFGDEGAVIEVLHRGSADLNARNKRRQTPLHIAVNKGHLQVVKTLLDFGCHPSLQDSEGDTPLHDAISKKRDDILAVLLEAGADVTITNNNGFNALHHAALRGNPSAMRVLLSKLPRPWIVDEKKDDGYTALHLAALNNHVEVAELLVHQGNANLDIQNVNQQTALHLAVERQHTQIVRLLVRAGAKLDIQDKDGDTPLHEALRHHTLSQLRQLQDMQDVGKVDAAWEPSKNTLIMGLGTQGAEKKSAASIACFLAANGADLSIRNKKGQSPLDLCPDPNLCKALAKCHKEKVSGQVGSRSPSMISNDSETLEECMVCSDMKRDTLFGPCGHIATCSLCSPRVKKCLICKEQVQSRTKIEECVVCSDKKAAVLFQPCGHMCACENCANLMKKCVQCRAVVERRVPFIMCCGGKSSEDATDDISSGNIPVLQKDKDNTNVNADVQKLQQQLQDIKEQTMCPVCLDRLKNMIFLCGHGTCQLCGDRMSECPICRKAIERRILLY",
# RB6I2_HUMAN
"MYGSARSVGKVEPSSQSPGRSPRLPRSPRLGHRRTNSTGGSSGSSVGGGSGKTLSMENIQSLNAAYATSGPMYLSDHENVGSETPKSTMTLGRSGGRLPYGVRMTAMGSSPNIASSGVASDTIAFGEHHLPPVSMASTVPHSLRQARDNTIMDLQTQLKEVLRENDLLRKDVEVKESKLSSSMNSIKTFWSPELKKERALRKDEASKITIWKEQYRVVQEENQHMQMTIQALQDELRIQRDLNQLFQQDSSSRTGEPCVAELTEENFQRLHAEHERQAKELFLLRKTLEEMELRIETQKQTLNARDESIKKLLEMLQSKGLSAKATEEDHERTRRLAEAEMHVHHLESLLEQKEKENSMLREEMHRRFENAPDSAKTKALQTVIEMKDSKISSMERGLRDLEEEIQMLKSNGALSTEEREEEMKQMEVYRSHSKFMKNKVEQLKEELSSKEAQWEELKKKAAGLQAEIGQVKQELSRKDTELLALQTKLETLTNQFSDSKQHIEVLKESLTAKEQRAAILQTEVDALRLRLEEKETMLNKKTKQIQDMAEEKGTQAGEIHDLKDMLDVKERKVNVLQKKIENLQEQLRDKEKQMSSLKERVKSLQADTTNTDTALTTLEEALAEKERTIERLKEQRDRDEREKQEEIDNYKKDLKDLKEKVSLLQGDLSEKEASLLDLKEHASSLASSGLKKDSRLKTLEIALEQKKEECLKMESQLKKAHEAALEARASPEMSDRIQHLEREITRYKDESSKAQAEVDRLLEILKEVENEKNDKDKKIAELERQVKDQNKKVANLKHKEQVEKKKSAQMLEEARRREDNLNDSSQQLQDSLRKKDDRIEELEEALRESVQITAEREMVLAQEESARTNAEKQVEELLMAMEKVKQELESMKAKLSSTQQSLAEKETHLTNLRAERRKHLEEVLEMKQEALLAAISEKDANIALLELSSSKKKTQEEVAALKREKDRLVQQLKQQTQNRMKLMADNYEDDHFKSSHSNQTNHKPSPDQIIQPLLELDQNRSKLKLYIGHLTTLCHDRDPLILRGLTPPASYNLDDDQAAWENELQKMTRGQLQDELEKGERDNAELQEFANAILQQIADHCPDILEQVVNALEESS",
# ZDH17_HUMAN
"MQREEGFNTKMADGPDEYDTEAGCVPLLHPEEIKPQSHYNHGYGEPLGRKTHIDDYSTWDIVKATQYGIYERCRELVEAGYDVRQPDKENVTLLHWAAINNRIDLVKYYISKGAIVDQLGGDLNSTPLHWATRQGHLSMVVQLMKYGADPSLIDGEGCSCIHLAAQFGHTSIVAYLIAKGQDVDMMDQNGMTPLMWAAYRTHSVDPTRLLLTFNVSVNLGDKYHKNTALHWAVLAGNTTVISLLLEAGANVDAQNIKGESALDLAKQRKNVWMINHLQEARQAKGYDNPSFLRKLKADKEFRQKVMLGTPFLVIWLVGFIADLNIDSWLIKGLMYGGVWATVQFLSKSFFDHSMHSALPLGIYLATKFWMYVTWFFWFWNDLNFLFIHLPFLANSVALFYNFGKSWKSDPGIIKATEEQKKKTIVELAETGSLDLSIFCSTCLIRKPVRSKHCGVCNRCIAKFDHHCPWVGNCVGAGNHRYFMGYLFFLLFMICWMIYGCISYWGLHCETTYTKDGFWTYITQIATCSPWMFWMFLNSVFHFMWVAVLLMCQMYQISCLGITTNERMNARRYKHFKVTTTSIESPFNHGCVRNIIDFFEFRCCGLFRPVIVDWTRQYTIEYDQISGSGYQLV",
# CBLN2_HUMAN
"MQAPGRGPLGLRLMMPGRRGALREPGGCGSCLGVALALLLLLLPACCPVRAQNDTEPIVLEGKCLVVCDSSPSADGAVTSSLGISVRSGSAKVAFSATRSTNHEPSEMSNRTMTIYFDQVLVNIGNHFDLASSIFVAPRKGIYSFSFHVVKVYNRQTIQVSLMQNGYPVISAFAGDQDVTREAASNGVLLLMEREDKVHLKLERGNLMGGWKYSTFSGFLVFPL",
# SYT12_HUMAN
"MAVDVAEYHLSVIKSPPGWEVGVYAAGALALLGIAAVSLWKLWTSGSFPSPSPFPNYDYRYLQQKYGESCAEAREKRVPAWNAQRASTRGPPSRKGSLSIEDTFESISELGPLELMGRELDLAPYGTLRKSQSADSLNSISSVSNTFGQDFTLGQVEVSMEYDTASHTLNVAVMQGKDLLEREEASFESCFMRVSLLPDEQIVGISRIQRNAYSIFFDEKFSIPLDPTALEEKSLRFSVFGIDEDERNVSTGVVELKLSVLDLPLQPFSGWLYLQDQNKAADAVGEILLSLSYLPTAERLTVVVVKAKNLIWTNDKTTADPFVKVYLLQDGRKMSKKKTAVKRDDPNPVFNEAMIFSVPAIVLQDLSLRVTVAESSSDGRGDNVGHVIIGPSASGMGTTHWNQMLATLRRPVSMWHAVRRN",
# SLIK4_HUMAN
"MFLWLFLILSALISSTNADSDISVEICNVCSCVSVENVLYVNCEKVSVYRPNQLKPPWSNFYHLNFQNNFLNILYPNTFLNFSHAVSLHLGNNKLQNIEGGAFLGLSALKQLHLNNNELKILRADTFLGIENLEYLQADYNLIKYIERGAFNKLHKLKVLILNDNLISFLPDNIFRFASLTHLDIRGNRIQKLPYIGVLEHIGRVVELQLEDNPWNCSCDLLPLKAWLENMPYNIYIGEAICETPSDLYGRLLKETNKQELCPMGTGSDFDVRILPPSQLENGYTTPNGHTTQTSLHRLVTKPPKTTNPSKISGIVAGKALSNRNLSQIVSYQTRVPPLTPCPAPCFCKTHPSDLGLSVNCQEKNIQSMSELIPKPLNAKKLHVNGNSIKDVDVSDFTDFEGLDLLHLGSNQITVIKGDVFHNLTNLRRLYLNGNQIERLYPEIFSGLHNLQYLYLEYNLIKEISAGTFDSMPNLQLLYLNNNLLKSLPVYIFSGAPLARLNLRNNKFMYLPVSGVLDQLQSLTQIDLEGNPWDCTCDLVALKLWVEKLSDGIVVKELKCETPVQFANIELKSLKNEILCPKLLNKPSAPFTSPAPAITFTTPLGPIRSPPGGPVPLSILILSILVVLILTVFVAFCLLVFVLRRNKKPTVKHEGLGNPDCGSMQLQLRKHDHKTNKKDGLSTEAFIPQTIEQMSKSHTCGLKESETGFMFSDPPGQKVVMRNVADKEKDLLHVDTRKRLSTIDELDELFPSRDSNVFIQNFLESKKEYNSIGVSGFEIRYPEKQPDKKSKKSLIGGNHSKIVVEQRKSEYFELKAKLQSSPDYLQVLEEQTALNKI",
# RHG12_HUMAN
"MKMADRSGKIIPGQVYIEVEYDYEYEAKDRKIVIKQGERYILVKKTNDDWWQVKPDENSKAFYVPAQYVKEVTRKALMPPVKQVAGLPNNSTKIMQSLHLQRSTENVNKLPELSSFGKPSSSVQGTGLIRDANQNFGPSYNQGQTVNLSLDLTHNNGKFNNDSHSPKVSSQNRTRSFGHFPGPEFLDVEKTSFSQEQSCDSAGEGSERIHQDSESGDELSSSSTEQIRATTPPNQGRPDSPVYANLQELKISQSALPPLPGSPAIQINGEWETHKDSSGRCYYYNRGTQERTWKPPRWTRDASISKGDFQNPGDQELLSSEENYYSTSYSQSDSQCGSPPRGWSEELDERGHTLYTSDYTNEKWLKHVDDQGRQYYYSADGSRSEWELPKYNASSQQQREIIKSRSLDRRLQEPIVLTKWRHSTIVLDTNDKESPTASKPCFPENESSPSSPKHQDTASSPKDQEKYGLLNVTKIAENGKKVRKNWLSSWAVLQGSSLLFTKTQGSSTSWFGSNQSKPEFTVDLKGATIEMASKDKSSKKNVFELKTRQGTELLIQSDNDTVINDWFKVLSSTINNQAVETDEGIEEEIPDSPGIEKHDKEKEQKDPKKLRSFKVSSIDSSEQKKTKKNLKKFLTRRPTLQAVREKGYIKDQVFGSNLANLCQRENGTVPKFVKLCIEHVEEHGLDIDGIYRVSGNLAVIQKLRFAVNHDEKLDLNDSKWEDIHVITGALKMFFRELPEPLFTFNHFNDFVNAIKQEPRQRVAAVKDLIRQLPKPNQDTMQILFRHLRRVIENGEKNRMTYQSIAIVFGPTLLKPEKETGNIAVHTVYQNQIVELILLELSSIFGR",
# KIBRA_HUMAN
"MPRPELPLPEGWEEARDFDGKVYYIDHTNRTTSWIDPRDRYTKPLTFADCISDELPLGWEEAYDPQVGDYFIDHNTKTTQIEDPRVQWRREQEHMLKDYLVVAQEALSAQKEIYQVKQQRLELAQQEYQQLHAVWEHKLGSQVSLVSGSSSSSKYDPEILKAEIATAKSRVNKLKREMVHLQHELQFKERGFQTLKKIDKKMSDAQGSYKLDEAQAVLRETKAIKKAITCGEKEKQDLIKSLAMLKDGFRTDRGSHSDLWSSSSSLESSSFPLPKQYLDVSSQTDISGSFGINSNNQLAEKVRLRLRYEEAKRRIANLKIQLAKLDSEAWPGVLDSERDRLILINEKEELLKEMRFISPRKWTQGEVEQLEMARKRLEKDLQAARDTQSKALTERLKLNSKRNQLVRELEEATRQVATLHSQLKSLSSSMQSLSSGSSPGSLTSSRGSLVASSLDSSTSASFTDLYYDPFEQLDSELQSKVEFLLLEGATGFRPSGCITTIHEDEVAKTQKAEGGGRLQALRSLSGTPKSMTSLSPRSSLSSPSPPCSPLMADPLLAGDAFLNSLEFEDPELSATLCELSLGNSAQERYRLEEPGTEGKQLGQAVNTAQGCGLKVACVSAAVSDESVAGDSGVYEASVQRLGASEAAAFDSDESEAVGATRIQIALKYDEKNKQFAILIIQLSNLSALLQQQDQKVNIRVAVLPCSESTTCLFRTRPLDASDTLVFNEVFWVSMSYPALHQKTLRVDVCTTDRSHLEECLGGAQISLAEVCRSGERSTRWYNLLSYKYLKKQSRELKPVGVMAPASGPASTDAVSALLEQTAVELEKRQEGRSSTQTLEDSWRYEETSENEAVAEEEEEEVEEEEGEEDVFTEKASPDMDGYPALKVDKETNTETPAPSPTVVRPKDRRVGTPSQGPFLRGSTIIRSKTFSPGPQSQYVCRLNRSDSDSSTLSKKPPFVRNSLERRSVRMKRPSSVKSLRSERLIRTSLDLELDLQATRTWHSQLTQEISVLKELKEQLEQAKSHGEKELPQWLREDERFRLLLRMLEKRQMDRAEHKGELQTDKMMRAAAKDVHRLRGQSCKEPPEVQSFREKMAFFTRPRMNIPALSADDV",
# SIR2_HUMAN
"MAEPDPSHPLETQAGKVQEAQDSDSDSEGGAAGGEADMDFLRNLFSQTLSLGSQKERLLDELTLEGVARYMQSERCRRVICLVGAGISTSAGIPDFRSPSTGLYDNLEKYHLPYPEAIFEISYFKKHPEPFFALAKELYPGQFKPTICHYFMRLLKDKGLLLRCYTQNIDTLERIAGLEQEDLVEAHGTFYTSHCVSASCRHEYPLSWMKEKIFSEVTPKCEDCQSLVKPDIVFFGESLPARFFSCMQSDFLKVDLLLVMGTSLQVQPFASLISKAPLSTPRLLINKEKAGQSDPFLGMIMGLGGGMDFDSKKAYRDVAWLGECDQGCLALAELLGWKKELEDLVRREHASIDAQSGAGVPNPSTSASPKKSPPPAKDEARTTEREKPQ",
# ABI1_HUMAN
"MAELQMLLEEEIPSGKRALIESYQNLTRVADYCENNYIQATDKRKALEETKAYTTQSLASVAYQINALANNVLQLLDIQASQLRRMESSINHISQTVDIHKEKVARREIGILTTNKNTSRTHKIIAPANMERPVRYIRKPIDYTVLDDVGHGVKWLKAKHGNNQPARTGTLSRTNPPTQKPPSPPMSGRGTLGRNTPYKTLEPVKPPTVPNDYMTSPARLGSQHSPGRTASLNQRPRTHSGSSGGSGSRENSGSSSIGIPIAVPTPSPPTIGPENISVPPPSGAPPAPPLAPLLPVSTVIAAPGSAPGSQYGTMTRQISRHNSTTSSTSSGGYRRTPSVTAQFSAQPHVNGGPLYSQNSISIAPPPPPMPQLTPQIPLTGFVARVQENIADSPTPPPPPPPDDIPMFDDSPPPPPPPPVDYEDEEAAVVQYNDPYADGDPAWAPKNYIEKVVAIYDYTKDKDDELSFMEGAIIYVIKKNDDGWYEGVCNRVTGLFPGNYVESIMHYTD",
# CA2D3_HUMAN
"MAGPGSPRRASRGASALLAAALLYAALGDVVRSEQQIPLSVVKLWASAFGGEIKSIAAKYSGSQLLQKKYKEYEKDVAIEEIDGLQLVKKLAKNMEEMFHKKSEAVRRLVEAAEEAHLKHEFDADLQYEYFNAVLINERDKDGNFLELGKEFILAPNDHFNNLPVNISLSDVQVPTNMYNKDPAIVNGVYWSESLNKVFVDNFDRDPSLIWQYFGSAKGFFRQYPGIKWEPDENGVIAFDCRNRKWYIQAATSPKDVVILVDVSGSMKGLRLTIAKQTVSSILDTLGDDDFFNIIAYNEELHYVEPCLNGTLVQADRTNKEHFREHLDKLFAKGIGMLDIALNEAFNILSDFNHTGQGSICSQAIMLITDGAVDTYDTIFAKYNWPDRKVRIFTYLIGREAAFADNLKWMACANKGFFTQISTLADVQENVMEYLHVLSRPKVIDQEHDVVWTEAYIDSTLPQAQKLTDDQGPVLMTTVAMPVFSKQNETRSKGILLGVVGTDVPVKELLKTIPKYKLGIHGYAFAITNNGYILTHPELRLLYEEGKKRRKPNYSSVDLSEVEWEDRDDVLRNAMVNRKTGKFSMEVKKTVDKGKRVLVMTNDYYYTDIKGTPFSLGVALSRGHGKYFFRGNVTIEEGLHDLEHPDVSLADEWSYCNTDLHPEHRHLSQLEAIKLYLKGKEPLLQCDKELIQEVLFDAVVSAPIEAYWTSLALNKSENSDKGVEVAFLGTRTGLSRINLFVGAEQLTNQDFLKAGDKENIFNADHFPLWYRRAAEQIPGSFVYSIPFSTGPVNKSNVVTASTSIQLLDERKSPVVAAVGIQMKLEFFQRKFWTASRQCASLDGKCSISCDDETVNCYLIDNNGFILVSEDYTQTGDFFGEIEGAVMNKLLTMGSFKRITLYDYQAMCRANKESSDGAHGLLDPYNAFLSAVKWIMTELVLFLVEFNLCSWWHSDMTAKAQKLKQTLEPCDTEYPAFVSERTIKETTGNIACEDCSKSFVIQQIPSSNLFMVVVDSSCLCESVAPITMAPIEIRYNESLKCERLKAQKIRRRPESCHGFHPEENARECGGAPSLQAQTVLLLLPLLLMLFSR",
# NLGNX_HUMAN
"MSRPQGLLWLPLLFTPVCVMLNSNVLLWLTALAIKFTLIDSQAQYPVVNTNYGKIRGLRTPLPNEILGPVEQYLGVPYASPPTGERRFQPPEPPSSWTGIRNTTQFAAVCPQHLDERSLLHDMLPIWFTANLDTLMTYVQDQNEDCLYLNIYVPTEDDIHDQNSKKPVMVYIHGGSYMEGTGNMIDGSILASYGNVIVITINYRLGILGFLSTGDQAAKGNYGLLDQIQALRWIEENVGAFGGDPKRVTIFGSGAGASCVSLLTLSHYSEGLFQKAIIQSGTALSSWAVNYQPAKYTRILADKVGCNMLDTTDMVECLRNKNYKELIQQTITPATYHIAFGPVIDGDVIPDDPQILMEQGEFLNYDIMLGVNQGEGLKFVDGIVDNEDGVTPNDFDFSVSNFVDNLYGYPEGKDTLRETIKFMYTDWADKENPETRRKTLVALFTDHQWVAPAVATADLHAQYGSPTYFYAFYHHCQSEMKPSWADSAHGDEVPYVFGIPMIGPTELFSCNFSKNDVMLSAVVMTYWTNFAKTGDPNQPVPQDTKFIHTKPNRFEEVAWSKYNPKDQLYLHIGLKPRVRDHYRATKVAFWLELVPHLHNLNEIFQYVSTTTKVPPPDMTSFPYGTRRSPAKIWPTTKRPAITPANNPKHSKDPHKTGPEDTTVLIETKRDYSTELSVTIAVGASLLFLNILAFAALYYKKDKRRHETHRRPSPQRNTTNDIAHIQNEEIMSLQMKQLEHDHECESLQAHDTLRLTCPPDYTLTLRRSPDDIPLMTPNTITMIPNTLTGMQPLHTFNTFSGGQNSTNLPHGHSTTRV",
# CADM3_HUMAN
"MGAPAASLLLLLLLFACCWAPGGANLSQDDSQPWTSDETVVAGGTVVLKCQVKDHEDSSLQWSNPAQQTLYFGEKRALRDNRIQLVTSTPHELSISISNVALADEGEYTCSIFTMPVRTAKSLVTVLGIPQKPIITGYKSSLREKDTATLNCQSSGSKPAARLTWRKGDQELHGEPTRIQEDPNGKTFTVSSSVTFQVTREDDGASIVCSVNHESLKGADRSTSQRIEVLYTPTAMIRPDPPHPREGQKLLLHCEGRGNPVPQQYLWEKEGSVPPLKMTQESALIFPFLNKSDSGTYGCTATSNMGSYKAYYTLNVNDPSPVPSSSSTYHAIIGGIVAFIVFLLLIMLIFLGHYLIRHKGTYLTHEAKGSDDAPDADTAIINAEGGQSGGDDKKEYFI",
# VPS52_HUMAN
"MAAAATMAAAARELVLRAGTSDMEEEEGPLAGGPGLQEPLQLGELDITSDEFILDEVDVHIQANLEDELVKEALKTGVDLRHYSKQVELELQQIEQKSIRDYIQESENIASLHNQITACDAVLERMEQMLGAFQSDLSSISSEIRTLQEQSGAMNIRLRNRQAVRGKLGELVDGLVVPSALVTAILEAPVTEPRFLEQLQELDAKAAAVREQEARGTAACADVRGVLDRLRVKAVTKIREFILQKIYSFRKPMTNYQIPQTALLKYRFFYQFLLGNERATAKEIRDEYVETLSKIYLSYYRSYLGRLMKVQYEEVAEKDDLMGVEDTAKKGFFSKPSLRSRNTIFTLGTRGSVISPTELEAPILVPHTAQRGEQRYPFEALFRSQHYALLDNSCREYLFICEFFVVSGPAAHDLFHAVMGRTLSMTLKHLDSYLADCYDAIAVFLCIHIVLRFRNIAAKRDVPALDRYWEQVLALLWPRFELILEMNVQSVRSTDPQRLGGLDTRPHYITRRYAEFSSALVSINQTIPNERTMQLLGQLQVEVENFVLRVAAEFSSRKEQLVFLINNYDMMLGVLMERAADDSKEVESFQQLLNARTQEFIEELLSPPFGGLVAFVKEAEALIERGQAERLRGEEARVTQLIRGFGSSWKSSVESLSQDVMRSFTNFRNGTSIIQGALTQLIQLYHRFHRVLSQPQLRALPARAELINIHHLMVELKKHKPNF",
# DOCK4_HUMAN
"MWIPTEHEKYGVVIASFRGTVPYGLSLEIGDTVQILEKCDGWYRGFALKNPNIKGIFPSSYVHLKNACVKNKGQFEMVIPTEDSVITEMTSTLRDWGTMWKQLYVRNEGDLFHRLWHIMNEILDLRRQVLVGHLTHDRMKDVKRHITARLDWGNEQLGLDLVPRKEYAMVDPEDISITELYRLMEHRHRKKDTPVQASSHHLFVQMKSLMCSNLGEELEVIFSLFDSKENRPISERFFLRLNRNGLPKAPDKPERHCSLFVDLGSSELRKDIYITVHIIRIGRMGAGEKKNACSVQYRRPFGCAVLSIADLLTGETKDDLILKVYMCNTESEWYQIHENIIKKLNARYNLTGSNAGLAVSLQLLHGDIEQIRREYSSVFSHGVSITRKLGFSNIIMPGEMRNDLYITIERGEFEKGGKSVARNVEVTMFIVDSSGQTLKDFISFGSGEPPASEYHSFVLYHNNSPRWSELLKLPIPVDKFRGAHIRFEFRHCSTKEKGEKKLFGFSFVPLMQEDGRTLPDGTHELIVHKCEENTNLQDTTRYLKLPFSKGIFLGNNNQAMKATKESFCITSFLCSTKLTQNGDMLDLLKWRTHPDKITGCLSKLKEIDGSEIVKFLQDTLDTLFGILDENSQKYGSKVFDSLVHIINLLQDSKFHHFKPVMDTYIESHFAGALAYRDLIKVLKWYVDRITEAERQEHIQEVLKAQEYIFKYIVQSRRLFSLATGGQNEEEFRCCIQELLMSVRFFLSQESKGSGALSQSQAVFLSSFPAVYSELLKLFDVREVANLVQDTLGSLPTILHVDDSLQAIKLQCIGKTVESQLYTNPDSRYILLPVVLHHLHIHLQEQKDLIMCARILSNVFCLIKKNSSEKSVLEEIDVIVASLLDILLRTILEITSRPQPSSSAMRFQFQDVTGEFVACLLSLLRQMTDRHYQQLLDSFNTKEELRDFLLQIFTVFRILIRPEMFPKDWTVMRLVANNVIITTVLYLSDALRKNFLNENFDYKIWDSYFYLAVIFINQLCLQLEMFTPSKKKKVLEKYGDMRVTMGCEIFSMWQNLGEHKLHFIPALIGPFLEVTLIPQPDLRNVMIPIFHDMMDWEQRRSGNFKQVEAKLIDKLDSLMSEGKGDETYRELFNSILLKKIERETWRESGVSLIATVTRLMERLLDYRDCMKMGEVDGKKIGCTVSLLNFYKTELNKEEMYIRYIHKLYDLHLKAQNFTEAAYTLLLYDELLEWSDRPLREFLTYPMQTEWQRKEHLHLTIIQNFDRGKCWENGIILCRKIAEQYESYYDYRNLSKMRMMEASLYDKIMDQQRLEPEFFRVGFYGKKFPFFLRNKEFVCRGHDYERLEAFQQRMLNEFPHAIAMQHANQPDETIFQAEAQYLQIYAVTPIPESQEVLQREGVPDNIKSFYKVNHIWKFRYDRPFHKGTKDKENEFKSLWVERTSLYLVQSLPGISRWFEVEKREVVEMSPLENAIEVLENKNQQLKTLISQCQTRQMQNINPLTMCLNGVIDAAVNGGVSRYQEAFFVKEYILSHPEDGEKIARLRELMLEQAQILEFGLAVHEKFVPQDMRPLHKKLVDQFFVMKSSLGIQEFSACMQASPVHFPNGSPRVCRNSAPASVSPDGTRVIPRRSPLSYPAVNRYSSSSLSSQASAEVSNITGQSESSDEVFNMQPSPSTSSLSSTHSASPNVTSSAPSSARASPLLSDKHKHSRENSCLSPRERPCSAIYPTPVEPSQRMLFNHIGDGALPRSDPNLSAPEKAVNPTPSSWSLDSGKEAKNMSDSGKLISPPVPPRPTQTASPARHTTSVSPSPAGRSPLKGSVQSFTPSPVEYHSPGLISNSPVLSGSYSSGISSLSRCSTSETSGFENQVNEQSAPLPVPVPVPVPSYGGEEPVRKESKTPPPYSVYERTLRRPVPLPHSLSIPVTSEPPALPPKPLAARSSHLENGARRTDPGPRPRPLPRKVSQL",
# NLGN1_HUMAN
"MALPRCTWPNYVWRAVMACLVHRGLGAPLTLCMLGCLLQAGHVLSQKLDDVDPLVATNFGKIRGIKKELNNEILGPVIQFLGVPYAAPPTGERRFQPPEPPSPWSDIRNATQFAPVCPQNIIDGRLPEVMLPVWFTNNLDVVSSYVQDQSEDCLYLNIYVPTEDVKRISKECARKPGKKICRKGGPLTKKQTDDLGDNDGAEDEDIRDSGGPKPVMVYIHGGSYMEGTGNLYDGSVLASYGNVIVITVNYRLGVLGFLSTGDQAAKGNYGLLDLIQALRWTSENIGFFGGDPLRITVFGSGAGGSCVNLLTLSHYSEGNRWSNSTKGLFQRAIAQSGTALSSWAVSFQPAKYARMLATKVGCNVSDTVELVECLQKKPYKELVDQDIQPARYHIAFGPVIDGDVIPDDPQILMEQGEFLNYDIMLGVNQGEGLKFVENIVDSDDGISASDFDFAVSNFVDNLYGYPEGKDVLRETIKFMYTDWADRHNPETRRKTLLALFTDHQWVAPAVATADLHSNFGSPTYFYAFYHHCQTDQVPAWADAAHGDEVPYVLGIPMIGPTELFPCNFSKNDVMLSAVVMTYWTNFAKTGDPNQPVPQDTKFIHTKPNRFEEVAWTRYSQKDQLYLHIGLKPRVKEHYRANKVNLWLELVPHLHNLNDISQYTSTTTKVPSTDITFRPTRKNSVPVTSAFPTAKQDDPKQQPSPFSVDQRDYSTELSVTIAVGASLLFLNILAFAALYYKKDKRRHDVHRRCSPQRTTTNDLTHAQEEEIMSLQMKHTDLDHECESIHPHEVVLRTACPPDYTLAMRRSPDDVPLMTPNTITMIPNTIPGIQPLHTFNTFTGGQNNTLPHPHPHPHSHSTTRV",
# ELMD1_HUMAN
"MKHFLRMLIQVCLYFYCKFLWRCLKFVMRKLTGRCELQRICYNTKPGASRTMKIETSLRDSKSKLLQTSVSVHPDAIEKTIEDIMELKKINPDVNPQLGISLQACLLQIVGYRNLIADVEKLRREAYDSDNPQHEEMLLKLWKFLKPNTPLESRISKQWCEIGFQGDDPKTDFRGMGLLGLYNLQYFAERDATAAQQVLSDSLHPKCRDITKEEISKFSKAEWEKKRMDKAIGYSFAIVGINITDLAYNLLVSGALKTHFYNIAPEAPTLSHFQQTFCYLMHEFHKFWIEEDPMDIMEFNRVREKFRKRIIKQLQNPDMALCPHFAASEGLINM",
# CADM2_HUMAN
"MIWKRSAVLRFYSVCGLLLQGSQGQFPLTQNVTVVEGGTAILTCRVDQNDNTSLQWSNPAQQTLYFDDKKALRDNRIELVRASWHELSISVSDVSLSDEGQYTCSLFTMPVKTSKAYLTVLGVPEKPQISGFSSPVMEGDLMQLTCKTSGSKPAADIRWFKNDKEIKDVKYLKEEDANRKTFTVSSTLDFRVDRSDDGVAVICRVDHESLNATPQVAMQVLEIHYTPSVKIIPSTPFPQEGQPLILTCESKGKPLPEPVLWTKDGGELPDPDRMVVSGRELNILFLNKTDNGTYRCEATNTIGQSSAEYVLIVHDVPNTLLPTTIIPSLTTATVTTTVAITTSPTTSATTSSIRDPNALAGQNGPDHALIGGIVAVVVFVTLCSIFLLGRYLARHKGTYLTNEAKGAEDAPDADTAIINAEGSQVNAEEKKEYFI",
# SYNPO_HUMAN
"MLGPHLPPPPLAPSEGRPTPCAFQIPDGSYRCLALEAEESSGEEGLQGEVGPTDLEEDEGVSRSGDDSACRVTQGTPQLPKALGIQPPSCSREEQGASQHDDRASQDWDVVKAGQMMTASPSPGPGPRVAQKPALGRSTSLTEKDLKEAKARSQQIAAQLTTPPSSNSRGVQLFNRRRQRVNEFTLESHGQRGQKPSQESLRVLPSSLPGHAPGLSLSSTSLPEPGPPRHPSPQSPDRGVPGHSMEGYSEEASLLRHLEKVASEEEEVPLVVYLKENAALLTANGLHLSQNREAQQSSPAPPPAEVHSPAADVNQNLASPSATLTTPTSNSSHNPPATDVNQNPPATVVPQSLPLSSIQQNSSEAQLPSNGTGPASKPSTLCADGQPQAPAEEVRCSTLLIDKVSTPATTTSTFSREATLIPSSRPPASDFMSSSLLIDIQPNTLVVSADQEMSGRAAATTPTKVYSEVHFTLAKPPSVVNRTARPFGIQAPGGTSQMERSPMLERRHFGEKAPAPQPPSLPDRSPRPQRHIMSRSPMVERRMMGQRSPASERRPLGNFTAPPTYTETLSTAPLASWVRSPPSYSVLYPSSDPKSSHLKGQAVPASKTGILEESMARRGSRKSMFTFVEKPKVTPNPDLLDLVQTADEKRRQRDQGEVGVEEEPFALGAEASNFQQEPAPRDRASPAAAEEVVPEWASCLKSPRIQAKPKPKPNQNLSEASGKGAELYARRQSRMEKYVIESSSHTPELARCPSPTMSLPSSWKYPTNAPGAFRVASRSPARTPPASLYHGYLPENGVLRPEPTKQPPYQLRPSLFVLSPIKEPAKVSPRAASPAKPSSLDLVPNLPKGALPPSPALPRPSRSSPGLYTSPGQDSLQPTAVSPPYGGDISPVSPSRAWSPRAKQAPRPSFSTRNAGIEAQVWKPSFCFK",
# PNKD_HUMAN
"MAAVVAATALKGRGARNARVLRGILAGATANKASHNRTRALQSHSSPEGKEEPEPLSPELEYIPRKRGKNPMKAVGLAWYSLYTRTWLGYLFYRQQLRRARNRYPKGHSKTQPRLFNGVKVLPIPVLSDNYSYLIIDTQAQLAVAVDPSDPRAVQASIEKEGVTLVAILCTHKHWDHSGGNRDLSRRHRDCRVYGSPQDGIPYLTHPLCHQDVVSVGRLQIRALATPGHTQGHLVYLLDGEPYKGPSCLFSGDLLFLSGCGRTFEGNAETMLSSLDTVLGLGDDTLLWPGHEYAEENLGFAGVVEPENLARERKMQWVQRQRLERKGTCPSTLGEERSYNPFLRTHCLALQEALGPGPGPTGDDDYSRAQLLEELRRLKDMHKSK",
# MINK1_HUMAN
"MGDPAPARSLDDIDLSALRDPAGIFELVEVVGNGTYGQVYKGRHVKTGQLAAIKVMDVTEDEEEEIKQEINMLKKYSHHRNIATYYGAFIKKSPPGNDDQLWLVMEFCGAGSVTDLVKNTKGNALKEDCIAYICREILRGLAHLHAHKVIHRDIKGQNVLLTENAEVKLVDFGVSAQLDRTVGRRNTFIGTPYWMAPEVIACDENPDATYDYRSDIWSLGITAIEMAEGAPPLCDMHPMRALFLIPRNPPPRLKSKKWSKKFIDFIDTCLIKTYLSRPPTEQLLKFPFIRDQPTERQVRIQLKDHIDRSRKKRGEKEETEYEYSGSEEEDDSHGEEGEPSSIMNVPGESTLRREFLRLQQENKSNSEALKQQQQLQQQQQRDPEAHIKHLLHQRQRRIEEQKEERRRVEEQQRREREQRKLQEKEQQRRLEDMQALRREEERRQAEREQEYKRKQLEEQRQSERLQRQLQQEHAYLKSLQQQQQQQQLQKQQQQQLLPGDRKPLYHYGRGMNPADKPAWAREVEERTRMNKQQNSPLAKSKPGSTGPEPPIPQASPGPPGPLSQTPPMQRPVEPQEGPHKSLVAHRVPLKPYAAPVPRSQSLQDQPTRNLAAFPASHDPDPAIPAPTATPSARGAVIRQNSDPTSEGPGPSPNPPAWVRPDNEAPPKVPQRTSSIATALNTSGAGGSRPAQAVRARPRSNSAWQIYLQRRAERGTPKPPGPPAQPPGPPNASSNPDLRRSDPGWERSDSVLPASHGHLPQAGSLERNRVGVSSKPDSSPVLSPGNKAKPDDHRSRPGRPADFVLLKERTLDEAPRPPKKAMDYSSSSEEVESSEDDEEEGEGGPAEGSRDTPGGRSDGDTDSVSTMVVHDVEEITGTQPPYGGGTMVVQRTPEEERNLLHADSNGYTNLPDVVQPSHSPTENSKGQSPPSKDGSGDYQSRGLVKAPGKSSFTMFVDLGIYQPGGSGDSIPITALVGGEGTRLDQLQYDVRKGSVVNVNPTNTRAHSETPEIRKYKKRFNSEILCAALWGVNLLVGTENGLMLLDRSGQGKVYGLIGRRRFQQMDVLEGLNLLITISGKRNKLRVYYLSWLRNKILHNDPEVEKKQGWTTVGDMEGCGHYRVVKYERIKFLVIALKSSVEVYAWAPKPYHKFMAFKSFADLPHRPLLVDLTVEEGQRLKVIYGSSAGFHAVDVDSGNSYDIYIPVHIQSQITPHAIIFLPNTDGMEMLLCYEDEGVYVNTYGRIIKDVVLQWGEMPTSVAYICSNQIMGWGEKAIEIRSVETGHLDGVFMHKRAQRLKFLCERNDKVFFASVRSGGSSQVYFMTLNRNCIMNW",
# SVOP_HUMAN
"MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLEAVELDDGAAVPKEFANPTDDTFMVEDAVEAIGFGKFQWKLSVLTGLAWMADAMEMMILSILAPQLHCEWRLPSWQVALLTSVVFVGMMSSSTLWGNISDQYGRKTGLKISVLWTLYYGILSAFAPVYSWILVLRGLVGFGIGGVPQSVTLYAEFLPMKARAKCILLIEVFWAIGTVFEVVLAVFVMPSLGWRWLLILSAVPLLLFAVLCFWLPESARYDVLSGNQEKAIATLKRIATENGAPMPLGKLIISRQEDRGKMRDLFTPHFRWTTLLLWFIWFSNAFSYYGLVLLTTELFQAGDVCGISSRKKAVEAKCSLACEYLSEEDYMDLLWTTLSEFPGVLVTLWIIDRLGRKKTMALCFVIFSFCSLLLFICVGRNVLTLLLFIARAFISGGFQAAYVYTPEVYPTATRALGLGTCSGMARVGALITPFIAQVMLESSVYLTLAVYSGCCLLAALASCFLPIETKGRGLQESSHREWGQEMVGRGMHGAGVTRSNSGSQE",
# RNF10_HUMAN
"MPLSSPNAAATASDMDKNSGSNSSSASSGSSKGQQPPRSASAGPAGESKPKSDGKNSSGSKRYNRKRELSYPKNESFNNQSRRSSSQKSKTFNKMPPQRGGGSSKLFSSSFNGGRRDEVAEAQRAEFSPAQFSGPKKINLNHLLNFTFEPRGQTGHFEGSGHGSWGKRNKWGHKPFNKELFLQANCQFVVSEDQDYTAHFADPDTLVNWDFVEQVRICSHEVPSCPICLYPPTAAKITRCGHIFCWACILHYLSLSEKTWSKCPICYSSVHKKDLKSVVATESHQYVVGDTITMQLMKREKGVLVALPKSKWMNVDHPIHLGDEQHSQYSKLLLASKEQVLHRVVLEEKVALEQQLAEEKHTPESCFIEAAIQELKTREEALSGLAGSRREVTGVVAALEQLVLMAPLAKESVFQPRKGVLEYLSAFDEETTEVCSLDTPSRPLALPLVEEEEAVSEPEPEGLPEACDDLELADDNLKEGTICTESSQQEPITKSGFTRLSSSPCYYFYQAEDGQHMFLHPVNVRCLVREYGSLERSPEKISATVVEIAGYSMSEDVRQRHRYLSHLPLTCEFSICELALQPPVVSKETLEMFSDDIEKRKRQRQKKAREERRRERRIEIEENKKQGKYPEVHIPLENLQQFPAFNSYTCSSDSALGPTSTEGHGALSISPLSRSPGSHADFLLTPLSPTASQGSPSFCVGSLEEDSPFPSFAQMLRVGKAKADVWPKTAPKKDENSLVPPAPVDSDGESDNSDRVPVPSFQNSFSQAIEAAFMKLDTPATSDPLSEEKGGKKRKKQKQKLLFSTSVVHTK",
# NGEF_HUMAN
"METRESEDLEKTRRKSASDQWNTDNEPAKVKPELLPEKEETSQADQDIQDKEPHCHIPIKRNSIFNRSIRRKSKAKARDNPERNASCLADSQDNGKSVNEPLTLNIPWSRMPPCRTAMQTDPGAQEMSESSSTPGNGATPEEWPALADSPTTLTEALRMIHPIPADSWRNLIEQIGLLYQEYRDKSTLQEIETRRQQDAEIEDNTNGSPASEDTPEEEEEEEEEEEPASPPERKTLPQICLLSNPHSRFNLWQDLPEIRSSGVLEILQPEEIKLQEAMFELVTSEASYYKSLNLLVSHFMENERIRKILHPSEAHILFSNVLDVLAVSERFLLELEHRMEENIVISDVCDIVYRYAADHFSVYITYVSNQTYQERTYKQLLQEKAAFRELIAQLELDPKCRGLPFSSFLILPFQRITRLKLLVQNILKRVEERSERECTALDAHKELEMVVKACNEGVRKMSRTEQMISIQKKMEFKIKSVPIISHSRWLLKQGELQQMSGPKTSRTLRTKKLFHEIYLFLFNDLLVICRQIPGDKYQVFDSAPRGLLRVEELEDQGQTLANVFILRLLENADDREATYMLKASSQSEMKRWMTSLAPNRRTKFVSFTSRLLDCPQVQCVHPYVAQQPDELTLELADILNILDKTDDGWIFGERLHDQERGWFPSSMTEEILNPKIRSQNLKECFRVHKMDDPQRSQNKDRRKLGSRNRQ",
# ARFG1_HUMAN
"MASPRTRKVLKEVRVQDENNVCFECGAFNPQWVSVTYGIWICLECSGRHRGLGVHLSFVRSVTMDKWKDIELEKMKAGGNAKFREFLESQEDYDPCWSLQEKYNSRAAALFRDKVVALAEGREWSLESSPAQNWTPPQPRTLPSMVHRVSGQPQSVTASSDKAFEDWLNDDLGSYQGAQGNRYVGFGNTPPPQKKEDDFLNNAMSSLYSGWSSFTTGASRFASAAKEGATKFGSQASQKASELGHSLNENVLKPAQEKVKEGKIFDDVSSGVSQLASKVQGVGSKGWRDVTTFFSGKAEGPLDSPSEGHSYQNSGLDHFQNSNIDQSFWETFGSAEPTKTRKSPSSDSWTCADTSTERRSSDSWEVWGSASTNRNSNSDGGEGGEGTKKAVPPAVPTDDGWDNQNW",
# ENAH_HUMAN
"MSEQSICQARAAVMVYDDANKKWVPAGGSTGFSRVHIYHHTGNNTFRVVGRKIQDHQVVINCAIPKGLKYNQATQTFHQWRDARQVYGLNFGSKEDANVFASAMMHALEVLNSQETGPTLPRQNSQLPAQVQNGPSQEELEIQRRQLQEQQRQKELERERLERERMERERLERERLERERLERERLEQEQLERERQERERQERLERQERLERQERLERQERLDRERQERQERERLERLERERQERERQEQLEREQLEWERERRISSAAAPASVETPLNSVLGDSSASEPGLQAASQPAETPSQQGIVLGPLAPPPPPPLPPGPAQASVALPPPPGPPPPPPLPSTGPPPPPPPPPLPNQVPPPPPPPPAPPLPASGFFLASMSEDNRPLTGLAAAIAGAKLRKVSRMEDTSFPSGGNAIGVNSASSKTDTGRGNGPLPLGGSGLMEEMSALLARRRRIAEKGSTIETEQKEDKGEDSEPVTSKASSTSTPEPTRKPWERTNTMNGSKSPVISRRDSPRKNQIVFDNRSYDSLHRPKSTPLSQPSANGVQTEGLDYDRLKQDILDEMRKELTKLKEELIDAIRQELSKSNTA",
# SYT2_HUMAN
"MRNIFKRNQEPIVAPATTTATMPIGPVDNSTESGGAGESQEDMFAKLKEKLFNEINKIPLPPWALIAIAVVAGLLLLTCCFCICKKCCCKKKKNKKEKGKGMKNAMNMKDMKGGQDDDDAETGLTEGEGEGEEEKEPENLGKLQFSLDYDFQANQLTVGVLQAAELPALDMGGTSDPYVKVFLLPDKKKKYETKVHRKTLNPAFNETFTFKVPYQELGGKTLVMAIYDFDRFSKHDIIGEVKVPMNTVDLGQPIEEWRDLQGGEKEEPEKLGDICTSLRYVPTAGKLTVCILEAKNLKKMDVGGLSDPYVKIHLMQNGKRLKKKKTTVKKKTLNPYFNESFSFEIPFEQIQKVQVVVTVLDYDKLGKNEAIGKIFVGSNATGTELRHWSDMLANPRRPIAQWHSLKPEEEVDALLGKNK",
# TDRD5_HUMAN
"MSEQERIQECLRKEIRSLLISTKDGLSPQELEKEYLLMVGNHLPLRILGYRSTMELVLDMPDVVRVCPGAGGTVILKAIPDESTKGIASLVAKQRSSHKLRNSMHKGRPSIYSGPRSHRRVPYRGRVAPILPAVVKSELKDLLALSPVLLSDFEKAFAKRFGRSFQYMQYGFLSMFEVLNAASDVISVEQTRAGSLLMLKKSVTEEKPRGCPAGKIFTQPFRMKQGSYSTGFPVAKPCFSQPTSNMEPPKQIMSMEKTSKLNVVETSRLNHTEKLNQLENTFKSVIAQIGPGGTISSELKHKIKFVVSKFPEGLFISKLLGEYEVIFKEQLSPKKLGFLNVTELVGALSDILHVEFRKGHQDLLVFDADKKPLPPVQSDKKIEAKACVSSPPRNSLSTAAVKETVWNCPSKKQKEPQQKICKKPNLVVKPLQLQVETNKSELNLAMANHDIPPDAVPNKKLCRLPPLDTSSLIGVFVEYIISPSQFYIRIYSRDSSELLEDMMIEMRRCYSNQLVSDRYVMPECFIQPGHLCCVRISEDKWWYRVIIHRVLEKQEVEVFYPDFGNIGIVQKSSLRFLKCCYTKLPAQAIPCSLAWVRPVEEHWTSKAILQFQKLCGLKPLVGVVDEYVDGILNIFLCDTSSNEDVYFHHVLRTEGHAIVCRENISSKGFSELNPLALYTTSSGGPEDIVLTELGYPSQQHYFNEDRKISPQSKESELRILDEIPTGMPCLESVTIGDDIWDENWLPLQAKMGKGGDAASHLFTASLGGKNQYSSCKEMPQKDWCFSTPKDTWDDSWQPSGLVNGTKVEVHKPEVLGAQEKNTGTNRTQKQLDINGSSDSSTLPKLEEFCTSLTQSEQSADGSQSEPNNSQTQPKQIQLSTAAPCSTTAVDDSAEKPSGSVESSPEILKNEDFSSSRAITLYKDKRQESVDQLSLILSYECQISQKLYIPRSTATAALGAAARLATSRSLLHWYPSVKRMEA",
# UN13C_HUMAN
"MVANFFKSLILPYIHKLCKGMFTKKLGNTNKNKEYRQQKKDQDFPTAGQTKSPKFSYTFKSTVKKIAKCSSTHNLSTEEDEASKEFSLSPTFSYRVAIANGLQKNAKVTNSDNEDLLQELSSIESSYSESLNELRSSTENQAQSTHTMPVRRNRKSSSSLAPSEGSSDGERTLHGLKLGALRKLRKWKKSQECVSSDSELSTMKKSWGIRSKSLDRTVRNPKTNALEPGFSSSGCISQTHDVMEMIFKELQGISQIETELSELRGHVNALKHSIDEISSSVEVVQSEIEQLRTGFVQSRRETRDIHDYIKHLGHMGSKASLRFLNVTEERFEYVESVVYQILIDKMGFSDAPNAIKIEFAQRIGHQRDCPNAKPRPILVYFETPQQRDSVLKKSYKLKGTGIGISTDILTHDIRERKEKGIPSSQTYESMAIKLSTPEPKIKKNNWQSPDDSDEDLESDLNRNSYAVLSKSELLTKGSTSKPSSKSHSARSKNKTANSSRISNKSDYDKISSQLPESDILEKQTTTHYADATPLWHSQSDFFTAKLSRSESDFSKLCQSYSEDFSENQFFTRTNGSSLLSSSDRELWQRKQEGTATLYDSPKDQHLNGGVQGIQGQTETENTETVDSGMSNGMVCASGDRSHYSDSQLSLHEDLSPWKEWNQGADLGLDSSTQEGFDYETNSLFDQQLDVYNKDLEYLGKCHSDLQDDSESYDLTQDDNSSPCPGLDNEPQGQWVGQYDSYQGANSNELYQNQNQLSMMYRSQSELQSDDSEDAPPKSWHSRLSIDLSDKTFSFPKFGSTLQRAKSALEVVWNKSTQSLSGYEDSGSSLMGRFRTLSQSTANESSTTLDSDVYTEPYYYKAEDEEDYTEPVADNETDYVEVMEQVLAKLENRTSITETDEQMQAYDHLSYETPYETPQDEGYDGPADDMVSEEGLEPLNETSAEMEIREDENQNIPEQPVEITKPKRIRPSFKEAALRAYKKQMAELEEKILAGDSSSVDEKARIVSGNDLDASKFSALQVCGGAGGGLYGIDSMPDLRRKKTLPIVRDVAMTLAARKSGLSLAMVIRTSLNNEELKMHVFKKTLQALIYPMSSTIPHNFEVWTATTPTYCYECEGLLWGIARQGMKCLECGVKCHEKCQDLLNADCLQRAAEKSSKHGAEDKTQTIITAMKERMKIREKNRPEVFEVIQEMFQISKEDFVQFTKAAKQSVLDGTSKWSAKITITVVSAQGLQAKDKTGSSDPYVTVQVGKNKRRTKTIFGNLNPVWDEKFYFECHNSTDRIKVRVWDEDDDIKSRVKQHFKKESDDFLGQTIVEVRTLSGEMDVWYNLEKRTDKSAVSGAIRLKINVEIKGEEKVAPYHIQYTCLHENLFHYLTEVKSNGGVKIPEVKGDEAWKVFFDDASQEIVDEFAMRYGIESIYQAMTHFSCLSSKYMCPGVPAVMSTLLANINAFYAHTTVSTNIQVSASDRFAATNFGREKFIKLLDQLHNSLRIDLSKYRENFPASNTERLQDLKSTVDLLTSITFFRMKVLELQSPPKASMVVKDCVRACLDSTYKYIFDNCHELYSQLTDPSKKQDIPREDQGPTTKNLDFWPQLITLMVTIIDEDKTAYTPVLNQFPQELNMGKISAEIMWTLFALDMKYALEEHENQRLCKSTDYMNLHFKVKWFYNEYVRELPAFKDAVPEYSLWFEPFVMQWLDENEDVSMEFLHGALGRDKKDGFQQTSEHALFSCSVVDVFAQLNQSFEIIKKLECPNPEALSHLMRRFAKTINKVLLQYAAIVSSDFSSHCDKENVPCILMNNIQQLRVQLEKMFESMGGKELDSEASTILKELQVKLSGVLDELSVTYGESFQVIIEECIKQMSFELNQMRANGNTTSNKNSAAMDAEIVLRSLMDFLDKTLSLSAKICEKTVLKRVLKELWKLVLNKIEKQIVLPPLTDQTGPQMIFIAAKDLGQLSKLKEHMIREDARGLTPRQCAIMEVVLATIKQYFHAGGNGLKKNFLEKSPDLQSLRYALSLYTQTTDALIKKFIDTQTSQSRSSKDAVGQISVHVDITATPGTGDHKVTVKVIAINDLNWQTTAMFRPFVEVCILGPNLGDKKRKQGTKTKSNTWSPKYNETFQFILGKENRPGAYELHLSVKDYCFAREDRIIGMTVIQLQNIAEKGSYGAWYPLLKNISMDETGLTILRILSQRTSDDVAKEFVRLKSETRSTEESA",
# ATAD1_HUMAN
"MVHAEAFSRPLSRNEVVGLIFRLTIFGAVTYFTIKWMVDAIDPTRKQKVEAQKQAEKLMKQIGVKNVKLSEYEMSIAAHLVDPLNMHVTWSDIAGLDDVITDLKDTVILPIKKKHLFENSRLLQPPKGVLLYGPPGCGKTLIAKATAKEAGCRFINLQPSTLTDKWYGESQKLAAAVFSLAIKLQPSIIFIDEIDSFLRNRSSSDHEATAMMKAQFMSLWDGLDTDHSCQVIVMGATNRPQDLDSAIMRRMPTRFHINQPALKQREAILKLILKNENVDRHVDLLEVAQETDGFSGSDLKEMCRDAALLCVREYVNSTSEESHDEDEIRPVQQQDLHRAIEKMKKSKDAAFQNVLTHVCLD",
# NETO2_HUMAN
"MALERLCSVLKVLLITVLVVEGIAVAQKTQDGQNIGIKHIPATQCGIWVRTSNGGHFASPNYPDSYPPNKECIYILEAAPRQRIELTFDEHYYIEPSFECRFDHLEVRDGPFGFSPLIDRYCGVKSPPLIRSTGRFMWIKFSSDEELEGLGFRAKYSFIPDPDFTYLGGILNPIPDCQFELSGADGIVRSSQVEQEEKTKPGQAVDCIWTIKATPKAKIYLRFLDYQMEHSNECKRNFVAVYDGSSSIENLKAKFCSTVANDVMLKTGIGVIRMWADEGSRLSRFRMLFTSFVEPPCTSSTFFCHSNMCINNSLVCNGVQNCAYPWDENHCKEKKKAGVFEQITKTHGTIIGITSGIVLVLLIISILVQVKQPRKKVMACKTAFNKTGFQEVFDPPHYELFSLRDKEISADLADLSEELDNYQKMRRSSTASRCIHDHHCGSQASSVKQSRTNLSSMELPFRNDFAQPQPMKTFNSTFKKSSYTFKQGHECPEQALEDRVMEEIPCEIYVRGREDSAQASISIDF",
# NECP1_HUMAN
"MATELEYESVLCVKPDVSVYRIPPRASNRGYRASDWKLDQPDWTGRLRITSKGKTAYIKLEDKVSGELFAQAPVEQYPGIAVETVTDSSRYFVIRIQDGTGRSAFIGIGFTDRGDAFDFNVSLQDHFKWVKQESEISKESQEMDARPKLDLGFKEGQTIKLCIGNITNKKGGASKPRTARGGGLSLLPPPPGGKVTIPPPSSSVAISNHVTPPPIPKSNHGGSDADILLDLDSPAPVTTPAPTPVSVSNDLWGDFSTASSSVPNQAPQPSNWVQF",
# CAMKV_HUMAN
"MPFGCVTLGDKKNYNQPSEVTDRYDLGQVIKTEEFCEIFRAKDKTTGKLHTCKKFQKRDGRKVRKAAKNEIGILKMVKHPNILQLVDVFVTRKEYFIFLELATGREVFDWILDQGYYSERDTSNVVRQVLEAVAYLHSLKIVHRNLKLENLVYYNRLKNSKIVISDFHLAKLENGLIKEPCGTPEYLAPEVVGRQRYGRPVDCWAIGVIMYILLSGNPPFYEEVEEDDYENHDKNLFRKILAGDYEFDSPYWDDISQAAKDLVTRLMEVEQDQRITAEEAISHEWISGNAASDKNIKDGVCAQIEKNFARAKWKKAVRVTTLMKRLRAPEQSSTAAAQSASATDTATPGAAGGATAAAASGATSAPEGDAARAAKSDNVAPADRSATPATDGSATPATDGSVTPATDGSITPATDGSVTPATDRSATPATDGRATPATEESTVPTTQSSAMLATKAAATPEPAMAQPDSTAPEGATGQAPPSSKGEEAAGYAQESQREEAS",
# CARL3_HUMAN
"MAKPSVELTRELQDSIRRCLSQGAVLQQHHVKLETKPKKFEDRVLALTSWRLHLFLLKVPAKVESSFNVLEIRAFNTLSQNQILVETERGMVSMRLPSAESVDQVTRHVSSALSKVCPGPGCLIRRGNADTPEGPRDTSPNSETSTSTTHSVCGGFSETYAALCDYNGLHCREEVQWDVDTIYHAEDNREFNLLDFSHLESRDLALMVAALAYNQWFTKLYCKDLRLGSEVLEQVLHTLSKSGSLEELVLDNAGLKTDFVQKLAGVFGENGSCVLHALTLSHNPIEDKGFLSLSQQLLCFPSGLTKLCLAKTAISPRGLQALGQTFGANPAFASSLRYLDLSKNPGLLATDEANALYSFLAQPNALVHLDLSGTDCVIDLLLGALLHGCCSHLTYLNLARNSCSHRKGREAPPAFKQFFSSAYTLSHVNLSATKLPLEALRALLQGLSLNSHLSDLHLDLSSCELRSAGAQALQEQLGAVTCVGSLDLSDNGFDSDLLTLVPALGKNKSLKHLFLGKNFNVKAKTLEEILHKLVQLIQEEDCSLQSLSVADSRLKLRTSILINALGSNTCLAKVDLSGNGMEDIGAKMLSKALQINSSLRTILWDRNNTSALGFLDIARALESNHTLRFMSFPVSDISQAYRSAPERTEDVWQKIQWCLVRNNHSQTCPQEQAFRLQQGLVTSSAEQMLQRLCGRVQEEVRALRLCPLEPVQDELLYARDLIKDAKNSRALFPSLYELGHVLANDGPVRQRLESVASEVSKAVDKELQVILESMVSLTQELCPVAMRVAEGHNKMLSNVAERVTVPRNFIRGALLEQAGQDIQNKLDEVKLSVVTYLTSSIVDEILQELYHSHKSLARHLTQLRTLSDPPGCPGQGQDLSSRGRGRNHDHEETTDDELGTNIDTMAIKKQKRCRKIRPVSAFISGSPQDMESQLGNLGIPPGWFSGLGGSQPTASGSWEGLSELPTHGYKLRHQTQGRPRPPRTTPPGPGRPSMPAPGTRQENGMATRLDEGLEDFFSRRVLEESSSYPRTLRTVRPGLSEAPLPPLQKKRRRGLFHFRRPRSFKGDRGPGSPTTGLLLPPPPPPPPTQESPPSPDPPSLGNNSSPCWSPEEESSLLPGFGGGRGPSFRRKMGTEGSEPGEGGPAPGTAQQPRVHGVALPGLERAKGWSFDGKREGPGPDQEGSTQAWQKRRSSDDAGPGSWKPPPPPQSTKPSFSAMRRAEATWHIAEESAPNHSCQSPSPASQDGEEEKEGTLFPERTLPARNAKLQDPALAPWPPKPVAVPRGRQPPQEPGVREEAEAGDAAPGVNKPRLRLSSQQDQEEPEVQGPPDPGRRTAPLKPKRTRRAQSCDKLEPDRRRPPDPTGTSEPGTD",
# LIPB2_HUMAN
"MASDASHALEAALEQMDGIIAGTKTGADLSDGTCEPGLASPASYMNPFPVLHLIEDLRLALEMLELPQERAALLSQIPGPTAAYIKEWFEESLSQVNHHSAASNETYQERLARLEGDKESLILQVSVLTDQVEAQGEKIRDLEVCLEGHQVKLNAAEEMLQQELLSRTSLETQKLDLMTEVSELKLKLVGMEKEQREQEEKQRKAEELLQELRHLKIKVEELENERNQYEWKLKATKAEVAQLQEQVALKDAEIERLHSQLSRTAALHSESHTERDQEIQRLKMGMETLLLANEDKDRRIEELTGLLNQYRKVKEIVMVTQGPSERTLSINEEEPEGGFSKWNATNKDPEELFKQEMPPRCSSPTVGPPPLPQKSLETRAQKKLSCSLEDLRSESVDKCMDGNQPFPVLEPKDSPFLAEHKYPTLPGKLSGATPNGEAAKSPPTICQPDATGSSLLRLRDTESGWDDTAVVNDLSSTSSGTESGPQSPLTPDGKRNPKGIKKFWGKIRRTQSGNFYTDTLGMAEFRRGGLRATAGPRLSRTRDSKGQKSDANAPFAQWSTERVCAWLEDFGLAQYVIFARQWVSSGHTLLTATPQDMEKELGIKHPLHRKKLVLAVKAINTKQEEKSALLDHIWVTRWLDDIGLPQYKDQFHESRVDRRMLQYLTVNDLLFLKVTSQLHHLSIKCAIHVLHVNKFNPHCLHRRPADESNLSPSEVVQWSNHRVMEWLRSVDLAEYAPNLRGSGVHGGLIILEPRFTGDTLAMLLNIPPQKTLLRRHLTTKFNALIGPEAEQEKREKMASPAYTPLTTTAKVRPRKLGFSHFGNIRKKKFDESTDYICPMEPSDGVSDSHRVYSGYRGLSPLDAPELDGLDQVGQIS",
# VGLU3_HUMAN
"MPFKAFDTFKEKILKPGKEGVKNAVGDSLGILQRKIDGTTEEEDNIELNEEGRPVQTSRPSPPLCDCHCCGLPKRYIIAIMSGLGFCISFGIRCNLGVAIVEMVNNSTVYVDGKPEIQTAQFNWDPETVGLIHGSFFWGYIMTQIPGGFISNKFAANRVFGAAIFLTSTLNMFIPSAARVHYGCVMCVRILQGLVEGVTYPACHGMWSKWAPPLERSRLATTSFCGSYAGAVVAMPLAGVLVQYIGWSSVFYIYGMFGIIWYMFWLLQAYECPAAHPTISNEEKTYIETSIGEGANVVSLSKFSTPWKRFFTSLPVYAIIVANFCRSWTFYLLLISQPAYFEEVFGFAISKVGLLSAVPHMVMTIVVPIGGQLADYLRSRQILTTTAVRKIMNCGGFGMEATLLLVVGFSHTKGVAISFLVLAVGFSGFAISGFNVNHLDIAPRYASILMGISNGVGTLSGMVCPLIVGAMTRHKTREEWQNVFLIAALVHYSGVIFYGVFASGEKQEWADPENLSEEKCGIIDQDELAEEIELNHESFASPKKKMSYGATSQNCEVQKKEWKGQRGATLDEEELTSYQNEERNFSTIS",
# CPEB3_HUMAN
"MQDDLLMDKSKTQPQPQQQQRQQQQPQPESSVSEAPSTPLSSETPKPEENSAVPALSPAAAPPAPNGPDKMQMESPLLPGLSFHQPPQQPPPPQEPAAPGASLSPSFGSTWSTGTTNAVEDSFFQGITPVNGTMLFQNFPHHVNPVFGGTFSPQIGLAQTQHHQQPPPPAPAPQPAQPAQPPQAQPPQQRRSPASPSQAPYAQRSAAAAYGHQPIMTSKPSSSSAVAAAAAAAAASSASSSWNTHQSVNAAWSAPSNPWGGLQAGRDPRRAVGVGVGVGVGVPSPLNPISPLKKPFSSNVIAPPKFPRAAPLTSKSWMEDNAFRTDNGNNLLPFQDRSRPYDTFNLHSLENSLMDMIRTDHEPLKGKHYPPSGPPMSFADIMWRNHFAGRMGINFHHPGTDNIMALNNAFLDDSHGDQALSSGLSSPTRCQNGERVERYSRKVFVGGLPPDIDEDEITASFRRFGPLVVDWPHKAESKSYFPPKGYAFLLFQEESSVQALIDACLEEDGKLYLCVSSPTIKDKPVQIRPWNLSDSDFVMDGSQPLDPRKTIFVGGVPRPLRAVELAMIMDRLYGGVCYAGIDTDPELKYPKGAGRVAFSNQQSYIAAISARFVQLQHNDIDKRVEVKPYVLDDQMCDECQGTRCGGKFAPFFCANVTCLQYYCEYCWASIHSRAGREFHKPLVKEGGDRPRHVPFRWS",
# PK3C3_HUMAN
"MGEAEKFHYIYSCDLDINVQLKIGSLEGKREQKSYKAVLEDPMLKFSGLYQETCSDLYVTCQVFAEGKPLALPVRTSYKAFSTRWNWNEWLKLPVKYPDLPRNAQVALTIWDVYGPGKAVPVGGTTVSLFGKYGMFRQGMHDLKVWPNVEADGSEPTKTPGRTSSTLSEDQMSRLAKLTKAHRQGHMVKVDWLDRLTFREIEMINESEKRSSNFMYLMVEFRCVKCDDKEYGIVYYEKDGDESSPILTSFELVKVPDPQMSMENLVESKHHKLARSLRSGPSDHDLKPNAATRDQLNIIVSYPPTKQLTYEEQDLVWKFRYYLTNQEKALTKFLKCVNWDLPQEAKQALELLGKWKPMDVEDSLELLSSHYTNPTVRRYAVARLRQADDEDLLMYLLQLVQALKYENFDDIKNGLEPTKKDSQSSVSENVSNSGINSAEIDSSQIITSPLPSVSSPPPASKTKEVPDGENLEQDLCTFLISRACKNSTLANYLYWYVIVECEDQDTQQRDPKTHEMYLNVMRRFSQALLKGDKSVRVMRSLLAAQQTFVDRLVHLMKAVQRESGNRKKKNERLQALLGDNEKMNLSDVELIPLPLEPQVKIRGIIPETATLFKSALMPAQLFFKTEDGGKYPVIFKHGDDLRQDQLILQIISLMDKLLRKENLDLKLTPYKVLATSTKHGFMQFIQSVPVAEVLDTEGSIQNFFRKYAPSENGPNGISAEVMDTYVKSCAGYCVITYILGVGDRHLDNLLLTKTGKLFHIDFGYILGRDPKPLPPPMKLNKEMVEGMGGTQSEQYQEFRKQCYTAFLHLRRYSNLILNLFSLMVDANIPDIALEPDKTVKKVQDKFRLDLSDEEAVHYMQSLIDESVHALFAAVVEQIHKFAQYWRK",
# NGDN_HUMAN
"MAALGVLESDLPSAVTLLKNLQEQVMAVTAQVKSLTQKVQAGAYPTEKGLSFLEVKDQLLLMYLMDLTHLILDKASGGSLQGHDAVLRLVEIRTVLEKLRPLDQKLKYQIDKLIKTAVTGSLSENDPLRFKPHPSNMMSKLSSEDEEEDEAEDDQSEASGKKSVKGVSKKYVPPRLVPVHYDETEAEREKKRLERAKRRALSSSVIRELKEQYSDAPEEIRDARHPHVTRQSQEDQHRINYEESMMVRLSVSKREKGRRKRANVMSSQLHSLTHFSDISALTGGTVHLDEDQNPIKKRKKIPQKGRKKKGFRRRR",
# TRPV1_HUMAN
"MKKWSSTDLGAAADPLQKDTCPDPLDGDPNSRPPPAKPQLSTAKSRTRLFGKGDSEEAFPVDCPHEEGELDSCPTITVSPVITIQRPGDGPTGARLLSQDSVAASTEKTLRLYDRRSIFEAVAQNNCQDLESLLLFLQKSKKHLTDNEFKDPETGKTCLLKAMLNLHDGQNTTIPLLLEIARQTDSLKELVNASYTDSYYKGQTALHIAIERRNMALVTLLVENGADVQAAAHGDFFKKTKGRPGFYFGELPLSLAACTNQLGIVKFLLQNSWQTADISARDSVGNTVLHALVEVADNTADNTKFVTSMYNEILMLGAKLHPTLKLEELTNKKGMTPLALAAGTGKIGVLAYILQREIQEPECRHLSRKFTEWAYGPVHSSLYDLSCIDTCEKNSVLEVIAYSSSETPNRHDMLLVEPLNRLLQDKWDRFVKRIFYFNFLVYCLYMIIFTMAAYYRPVDGLPPFKMEKTGDYFRVTGEILSVLGGVYFFFRGIQYFLQRRPSMKTLFVDSYSEMLFFLQSLFMLATVVLYFSHLKEYVASMVFSLALGWTNMLYYTRGFQQMGIYAVMIEKMILRDLCRFMFVYIVFLFGFSTAVVTLIEDGKNDSLPSESTSHRWRGPACRPPDSSYNSLYSTCLELFKFTIGMGDLEFTENYDFKAVFIILLLAYVILTYILLLNMLIALMGETVNKIAQESKNIWKLQRAITILDTEKSFLKCMRKAFRSGKLLQVGYTPDGKDDYRWCFRVDEVNWTTWNTNVGIINEDPGNCEGVKRTLSFSLRSSRVSGRHWKNFALVPLLREASARDRQSAQPEEVYLRQFSGSLKPEDAEVFKSPAASGEK",
# SYNE1_HUMAN
"MATSRGASRCPRDIANVMQRLQDEQEIVQKRTFTKWINSHLAKRKPPMVVDDLFEDMKDGVKLLALLEVLSGQKLPCEQGRRMKRIHAVANIGTALKFLEGRKIKLVNINSTDIADGRPSIVLGLMWTIILYFQIEELTSNLPQLQSLSSSASSVDSIVSSETPSPPSKRKVTTKIQGNAKKALLKWVQYTAGKQTGIEVKDFGKSWRSGVAFHSVIHAIRPELVDLETVKGRSNRENLEDAFTIAETELGIPRLLDPEDVDVDKPDEKSIMTYVAQFLKHYPDIHNASTDGQEDDEILPGFPSFANSVQNFKREDRVIFKEMKVWIEQFERDLTRAQMVESNLQDKYQSFKHFRVQYEMKRKQIEHLIQPLHRDGKLSLDQALVKQSWDRVTSRLFDWHIQLDKSLPAPLGTIGAWLYRAEVALREEITVQQVHEETANTIQRKLEQHKDLLQNTDAHKRAFHEIYRTRSVNGIPVPPDQLEDMAERFHFVSSTSELHLMKMEFLELKYRLLSLLVLAESKLKSWIIKYGRRESVEQLLQNYVSFIENSKFFEQYEVTYQILKQTAEMYVKADGSVEEAENVMKFMNETTAQWRNLSVEVRSVRSMLEEVISNWDRYGNTVASLQAWLEDAEKMLNQSENAKKDFFRNLPHWIQQHTAMNDAGNFLIETCDEMVSRDLKQQLLLLNGRWRELFMEVKQYAQADEMDRMKKEYTDCVVTLSAFATEAHKKLSEPLEVSFMNVKLLIQDLEDIEQRVPVMDAQYKIITKTAHLITKESPQEEGKEMFATMSKLKEQLTKVKECYSPLLYESQQLLIPLEELEKQMTSFYDSLGKINEIITVLEREAQSSALFKQKHQELLACQENCKKTLTLIEKGSQSVQKFVTLSNVLKHFDQTRLQRQIADIHVAFQSMVKKTGDWKKHVETNSRLMKKFEESRAELEKVLRIAQEGLEEKGDPEELLRRHTEFFSQLDQRVLNAFLKACDELTDILPEQEQQGLQEAVRKLHKQWKDLQGEAPYHLLHLKIDVEKNRFLASVEECRTELDRETKLMPQEGSEKIIKEHRVFFSDKGPHHLCEKRLQLIEELCVKLPVRDPVRDTPGTCHVTLKELRAAIDSTYRKLMEDPDKWKDYTSRFSEFSSWISTNETQLKGIKGEAIDTANHGEVKRAVEEIRNGVTKRGETLSWLKSRLKVLTEVSSENEAQKQGDELAKLSSSFKALVTLLSEVEKMLSNFGDCVQYKEIVKNSLEELISGSKEVQEQAEKILDTENLFEAQQLLLHHQQKTKRISAKKRDVQQQIAQAQQGEGGLPDRGHEELRKLESTLDGLERSRERQERRIQVTLRKWERFETNKETVVRYLFQTGSSHERFLSFSSLESLSSELEQTKEFSKRTESIAVQAENLVKEASEIPLGPQNKQLLQQQAKSIKEQVKKLEDTLEEDIKTMEMVKTKWDHFGSNFETLSVWITEKEKELNALETSSSAMDMQISQIKVTIQEIESKLSSIVGLEEEAQSFAQFVTTGESARIKAKLTQIRRYGEELREHAQCLEGTILGHLSQQQKFEENLRKIQQSVSEFEDKLAVPIKICSSATETYKVLQEHMDLCQALESLSSAITAFSASARKVVNRDSCVQEAAALQQQYEDILRRAKERQTALENLLAHWQRLEKELSSFLTWLERGEAKASSPEMDISADRVKVEGELQLIQALQNEVVSQASFYSKLLQLKESLFSVASKDDVKMMKLHLEQLDERWRDLPQIINKRINFLQSVVAEHQQFDELLLSFSVWIKLFLSELQTTSEISIMDHQVALTRHKDHAAEVESKKGELQSLQGHLAKLGSLGRAEDLHLLQGKAEDCFQLFEEASQVVERRQLALSHLAEFLQSHASLSGILRQLRQTVEATNSMNKNESDLIEKDLNDALQNAKALESAAVSLDGILSKAQYHLKIGSSEQRTSCRATADQLCGEVERIQNLLGTKQSEADALAVLKKAFQDQKEELLKSIEDIEERTDKERLKEPTRQALQQRLRVFNQLEDELNSHEHELCWLKDKAKQIAQKDVAFAPEVDREINRLEVTWDDTKRLIHENQGQCCGLIDLMREYQNLKSAVSKVLENASSVIVTRTTIKDQEDLKWAFSKHETAKNKMNYKQKDLDNFTSKGKHLLSELKKIHSSDFSLVKTDMESTVDKWLDVSEKLEENMDRLRVSLSIWDDVLSTRDEIEGWSNNCVPQMAENISNLDNHLRAEELLKEFESEVKNKALRLEELHSKVNDLKELTKNLETPPDLQFIEADLMQKLEHAKEITEVAKGTLKDFTAQSTQVEKFINDITTWFTKVEESLMNCAQNETCEALKKVKDIQKELQSQQSNISSTQENLNSLCRKYHSAELESLGRAMTGLIKKHEAVSQLCSKTQASLQESLEKHFSESMQEFQEWFLGAKAAAKESSDRTGDSKVLEAKLHDLQNILDSVSDGQSKLDAVTQEGQTLYAHLSKQIVSSIQEQITKANEEFQAFLKQCLKDKQALQDCASELGSFEDQHRKLNLWIHEMEERFNTENLGESKQHIPEKKNEVHKVEMFLGELLAARESLDKLSQRGQLLSEEGHGAGQEGRLCSQLLTSHQNLLRMTKEKLRSCQVALQEHEALEEALQSMWFWVKAIQDRLACAESTLGSKDTLEKRLSQIQDILLMKGEGEVKLNMAIGKGEQALRSSNKEGQRVIQTQLETLKEVWADIMSSSVHAQSTLESVISQWNDYVERKNQLEQWMESVDQKIEHPLQPQPGLKEKFVLLDHLQSILSEAEDHTRALHRLIAKSRELYEKTEDESFKDTAQEELKTQFNDIMTVAKEKMRKVEEIVKDHLMYLDAVHEFTDWLHSAKEELHRWSDMSGDSSATQKKLSKIKELIDSREIGASRLSRVESLAPEVKQNTTASGCELMHTEMQALRADWKQWEDSVFQTQSCLENLVSQMALSEQEFSGQVAQLEQALEQFSALLKTWAQQLTLLEGKNTDEEIVECWHKGQEILDALQKAEPRTEDLKSQLNELCRFSRDLSTYSGKVSGLIKEYNCLCLQASKGCQNKEQILQQRFRKAFRDFQQWLVNAKITTAKCFDIPQNISEVSTSLQKIQEFLSESENGQHKLNMMLSKGELLSTLLTKEKAKGIQAKVTAAKEDWKNFHSNLHQKESALENLKIQMKDFEVSAEPIQDWLSKTEKMVHESSNRLYDLPAKRREQQKLQSVLEEIHCYEPQLNRLKEKAQQLWEGQAASKSFRHRVSQLSSQYLALSNLTKEKVSRLDRIVAEHNQFSLGIKELQDWMTDAIHMLDSYCHPTSDKSVLDSRTLKLEALLSVKQEKEIQMKMIVTRGESVLQNTSPEGIPTIQQQLQSVKDMWASLLSAGIRCKSQLEGALSKWTSYQDGVRQFSGWMDSMEANLNESERQHAELRDKTTMLGKAKLLNEEVLSYSSLLETIEVKGAGMTEHYVTQLELQDLQERYRAIQERAKEAVTKSEKLVRLHQEYQRDLKAFEVWLGQEQEKLDQYSVLEGDAHTHETTLRDLQELQVHCAEGQALLNSVLHTREDVIPSGIPQAEDRALESLRQDWQAYQHRLSETRTQFNNVVNKLRLMEQKFQQVDEWLKTAEEKVSPRTRRQSNRATKEIQLHQMKKWHEEVTAYRDEVEEVGARAQEILDESHVNSRMGCQATQLTSRYQALLLQVLEQIKFLEEEIQSLEESESSLSSYSDWYGSTHKNFKNVATKIDKVDTVMMGKKLKTLEVLLKDMEKGHSLLKSAREKGERAVKYLEEGEAERLRKEIHDHMEQLKELTSTVRKEHMTLEKGLHLAKEFSDKCKALTQWIAEYQEILHVPEEPKMELYEKKAQLSKYKSLQQTVLSHEPSVKSVREKGEALLELVQDVTLKDKIDQLQSDYQDLCSIGKEHVFSLEAKVKDHEDYNSELQEVEKWLLQMSGRLVAPDLLETSSLETITQQLAHHKAMMEEIAGFEDRLNNLQMKGDTLIGQCADHLQAKLKQNVHAHLQGTKDSYSAICSTAQRMYQSLEHELQKHVSRQDTLQQCQAWLSAVQPDLEPSPQPPLSRAEAIKQVKHFRALQEQARTYLDLLCSMCDLSNASVKTTAKDIQQTEQTIEQKLVQAQNLTQGWEEIKHLKSELWIYLQDADQQLQNMKRRHSELELNIAQNMVSQVKDFVKKLQSKQASVNTIIEKVNKLTKKEESPEHKEINHLNDQWLDLCRQSNNLCLQREEDLQRTRDYHDCMNVVEVFLEKFTTEWDNLARSDAESTAVHLEALKKLALALQERKYAIEDLKDQKQKMIEHLNLDDKELVKEQTSHLEQRWFQLEDLIKRKIQVSVTNLEELNVVQSRFQELMEWAEEQQPNIAEALKQSPPPDMAQNLLMDHLAICSELEAKQMLLKSLIKDADRVMADLGLNERQVIQKALSDAQSHVNCLSDLVGQRRKYLNKALSEKTQFLMAVFQATSQIQQHERKIMFREHICLLPDDVSKQVKTCKSAQASLKTYQNEVTGLWAQGRELMKEVTEQEKSEVLGKLQELQSVYDSVLQKCSHRLQELEKNLVSRKHFKEDFDKACHWLKQADIVTFPEINLMNESSELHTQLAKYQNILEQSPEYENLLLTLQRTGQTILPSLNEVDHSYLSEKLNALPRQFNVIVALAKDKFYKVQEAILARKEYASLIELTTQSLSELEAQFLRMSKVPTDLAVEEALSLQDGCRAILDEVAGLGEAVDELNQKKEGFRSTGQPWQPDKMLHLVTLYHRLKRQTEQRVSLLEDTTSAYQEHEKMCQQLERQLKSVKEEQSKVNEETLPAEEKLKMYHSLAGSLQDSGIVLKRVTIHLEDLAPHLDPLAYEKARHQIQSWQGELKLLTSAIGETVTECESRMVQSIDFQTEMSRSLDWLRRVKAELSGPVYLDLNLQDIQEEIRKIQIHQEEVQSSLRIMNALSHKEKEKFTKAKELISADLEHSLAELSELDGDIQEALRTRQATLTEIYSQCQRYYQVFQAANDWLEDAQELLQLAGNGLDVESAEENLKSHMEFFSTEDQFHSNLEELHSLVATLDPLIKPTGKEDLEQKVASLELRSQRMSRDSGAQVDLLQRCTAQWHDYQKAREEVIELMNDTEKKLSEFSLLKTSSSHEAEEKLSEHKALVSVVNSFHEKIVALEEKASQLEKTGNDASKATLSRSMTTVWQRWTRLRAVAQDQEKILEDAVDEWTGFNNKVKKATEMIDQLQDKLPGSSAEKASKAELLTLLEYHDTFVLELEQQQSALGMLRQQTLSMLQDGAAPTPGEEPPLMQEITAMQDRCLNMQEKVKTNGKLVKQELKDREMVETQINSVKCWVQETKEYLGNPTIEIDAQLEELQILLTEATNHRQNIEKMAEEQKEKYLGLYTILPSELSLQLAEVALDLKIRDQIQDKIKEVEQSKATSQELSRQIQKLAKDLTTILTKLKAKTDNVVQAKTDQKVLGEELDGCNSKLMELDAAVQKFLEQNGQLGKPLAKKIGKLTELHQQTIRQAENRLSKLNQAASHLEEYNEMLELILKWIEKAKVLAHGTIAWNSASQLREQYILHQTLLEESKEIDSELEAMTEKLQYLTSVYCTEKMSQQVAELGRETEELRQMIKIRLQNLQDAAKDMKKFEAELKKLQAALEQAQATLTSPEVGRLSLKEQLSHRQHLLSEMESLKPKVQAVQLCQSALRIPEDVVASLPLCHAALRLQEEASRLQHTAIQQCNIMQEAVVQYEQYEQEMKHLQQLIEGAHREIEDKPVATSNIQELQAQISRHEELAQKIKGYQEQIASLNSKCKMLTMKAKHATMLLTVTEVEGLAEGTEDLDGELLPTPSAHPSVVMMTAGRCHTLLSPVTEESGEEGTNSEISSPPACRSPSPVANTDASVNQDIAYYQALSAERLQTDAAKIHPSTSASQEFYEPGLEPSATAKLGDLQRSWETLKNVISEKQRTLYEALERQQKYQDSLQSISTKMEAIELKLSESPEPGRSPESQMAEHQALMDEILMLQDEINELQSSLAEELVSESCEADPAEQLALQSTLTVLAERMSTIRMKASGKRQLLEEKLNDQLEEQRQEQALQRYRCEADELDSWLLSTKATLDTALSPPKEPMDMEAQLMDCQNMLVEIEQKVVALSELSVHNENLLLEGKAHTKDEAEQLAGKLRRLKGSLLELQRALHDKQLNMQGTAQEKEESDVDLTATQSPGVQEWLAQARTTWTQQRQSSLQQQKELEQELAEQKSLLRSVASRGEEILIQHSAAETSGDAGEKPDVLSQELGMEGEKSSAEDQMRMKWESLHQEFSTKQKLLQNVLEQEQEQVLYSRPNRLLSGVPLYKGDVPTQDKSAVTSLLDGLNQAFEEVSSQSGGAKRQSIHLEQKLYDGVSATSTWLDDVEERLFVATALLPEETETCLFNQEILAKDIKEMSEEMDKNKNLFSQAFPENGDNRDVIEDTLGCLLGRLSLLDSVVNQRCHQMKERLQQILNFQNDLKVLFTSLADNKYIILQKLANVFEQPVAEQIEAIQQAEDGLKEFDAGIIELKRRGDKLQVEQPSMQELSKLQDMYDELMMIIGSRRSGLNQNLTLKSQYERALQDLADLLETGQEKMAGDQKIIVSSKEEIQQLLDKHKEYFQGLESHMILTETLFRKIISFAVQKETQFHTELMAQASAVLKRAHKRGVELEYILETWSHLDEDQQELSRQLEVVESSIPSVGLVEENEDRLIDRITLYQHLKSSLNEYQPKLYQVLDDGKRLLISISCSDLESQLNQLGECWLSNTNKMSKELHRLETILKHWTRYQSESADLIHWLQSAKDRLEFWTQQSVTVPQELEMVRDHLNAFLEFSKEVDAQSSLKSSVLSTGNQLLRLKKVDTATLRSELSRIDSQWTDLLTNIPAVQEKLHQLQMDKLPSRHAISEVMSWISLMENVIQKDEDNIKNSIGYKAIHEYLQKYKGFKIDINCKQLTVDFVNQSVLQISSQDVESKRSDKTDFAEQLGAMNKSWQILQGLVTEKIQLLEGLLESWSEYENNVQCLKTWFETQEKRLKQQHRIGDQASVQNALKDCQDLEDLIKAKEKEVEKIEQNGLALIQNKKEDVSSIVMSTLRELGQTWANLDHMVGQLKILLKSVLDQWSSHKVAFDKINSYLMEARYSLSRFRLLTGSLEAVQVQVDNLQNLQDDLEKQERSLQKFGSITNQLLKECHPPVTETLTNTLKEVNMRWNNLLEEIAEQLQSSKALLQLWQRYKDYSKQCASTVQQQEDRTNELLKAATNKDIADDEVATWIQDCNDLLKGLGTVKDSLFFLHELGEQLKQQVDASAASAIQSDQLSLSQHLCALEQALCKQQTSLQAGVLDYETFAKSLEALEAWIVEAEEILQGQDPSHSSDLSTIQERMEELKGQMLKFSSMAPDLDRLNELGYRLPLNDKEIKRMQNLNRHWSLISSQTTERFSKLQSFLLQHQTFLEKCETWMEFLVQTEQKLAVEISGNYQHLLEQQRAHELFQAEMFSRQQILHSIIIDGQRLLEQGQVDDRDEFNLKLTLLSNQWQGVIRRAQQRRGIIDSQIRQWQRYREMAEKLRKWLVEVSYLPMSGLGSVPIPLQQARTLFDEVQFKEKVFLRQQGSYILTVEAGKQLLLSADSGAEAALQAELAEIQEKWKSASMRLEEQKKKLAFLLKDWEKCEKGIADSLEKLRTFKKKLSQSLPDHHEELHAEQMRCKELENAVGSWTDDLTQLSLLKDTLSAYISADDISILNERVELLQRQWEELCHQLSLRRQQIGERLNEWAVFSEKNKELCEWLTQMESKVSQNGDILIEEMIEKLKKDYQEEIAIAQENKIQLQQMGERLAKASHESKASEIEYKLGKVNDRWQHLLDLIAARVKKLKETLVAVQQLDKNMSSLRTWLAHIESELAKPIVYDSCNSEEIQRKLNEQQELQRDIEKHSTGVASVLNLCEVLLHDCDACATDAECDSIQQATRNLDRRWRNICAMSMERRLKIEETWRLWQKFLDDYSRFEDWLKSSERTAAFPSSSGVIYTVAKEELKKFEAFQRQVHECLTQLELINKQYRRLARENRTDSACSLKQMVHEGNQRWDNLQKRVTSILRRLKHFIGQREEFETARDSILVWLTEMDLQLTNIEHFSECDVQAKIKQLKAFQQEISLNHNKIEQIIAQGEQLIEKSEPLDAAIIEEELDELRRYCQEVFGRVERYHKKLIRLPLPDDEHDLSDRELELEDSAALSDLHWHDRSADSLLSPQPSSNLSLSLAQPLRSERSGRDTPASVDSIPLEWDHDYDLSRDLESAMSRALPSEDEEGQDDKDFYLRGAVGLSGDHSALESQIRQLGKALDDSRFQIQQTENIIRSKTPTGPELDTSYKGYMKLLGECSSSIDSVKRLEHKLKEEEESLPGFVNLHSTETQTAGVIDRWELLQAQALSKELRMKQNLQKWQQFNSDLNSIWAWLGDTEEELEQLQRLELSTDIQTIELQIKKLKELQKAVDHRKAIILSINLCSPEFTQADSKESRDLQDRLSQMNGRWDRVCSLLEEWRGLLQDALMQCQGFHEMSHGLLLMLENIDRRKNEIVPIDSNLDAEILQDHHKQLMQIKHELLESQLRVASLQDMSCQLLVNAEGTDCLEAKEKVHVIGNRLKLLLKEVSRHIKELEKLLDVSSSQQDLSSWSSADELDTSGSVSPTSGRSTPNRQKTPRGKCSLSQPGPSVSSPHSRSTKGGSDSSLSEPGPGRSGRGFLFRVLRAALPLQLLLLLLIGLACLVPMSEEDYSCALSNNFARSFHPMLRYTNGPPPL",
# MDGA1_HUMAN
"MEVTCLLLLALIPFHCRGQGVYAPAQAQIVHAGQACVVKEDNISERVYTIREGDTLMLQCLVTGHPRPQVRWTKTAGSASDKFQETSVFNETLRIERIARTQGGRYYCKAENGVGVPAIKSIRVDVQYLDEPMLTVHQTVSDVRGNFYQEKTVFLRCTVNSNPPARFIWKRGSDTLSHSQDNGVDIYEPLYTQGETKVLKLKNLRPQDYASYTCQVSVRNVCGIPDKAITFRLTNTTAPPALKLSVNETLVVNPGENVTVQCLLTGGDPLPQLQWSHGPGPLPLGALAQGGTLSIPSVQARDSGYYNCTATNNVGNPAKKTVNLLVRSMKNATFQITPDVIKESENIQLGQDLKLSCHVDAVPQEKVTYQWFKNGKPARMSKRLLVTRNDPELPAVTSSLELIDLHFSDYGTYLCMASFPGAPVPDLSVEVNISSETVPPTISVPKGRAVVTVREGSPAELQCEVRGKPRPPVLWSRVDKEAALLPSGLPLEETPDGKLRLERVSRDMSGTYRCQTARYNGFNVRPREAQVQLNVQFPPEVEPSSQDVRQALGRPVLLRCSLLRGSPQRIASAVWRFKGQLLPPPPVVPAAAEAPDHAELRLDAVTRDSSGSYECSVSNDVGSAACLFQVSAKAYSPEFYFDTPNPTRSHKLSKNYSYVLQWTQREPDAVDPVLNYRLSIRQLNQHNAVVKAIPVRRVEKGQLLEYILTDLRVPHSYEVRLTPYTTFGAGDMASRIIHYTEPINSPNLSDNTCHFEDEKICGYTQDLTDNFDWTRQNALTQNPKRSPNTGPPTDISGTPEGYYMFIETSRPRELGDRARLVSPLYNASAKFYCVSFFYHMYGKHIGSLNLLVRSRNKGALDTHAWSLSGNKGNVWQQAHVPISPSGPFQIIFEGVRGPGYLGDIAIDDVTLKKGECPRKQTDPNKVVVMPGSGAPCQSSPQLWGPMAIFLLALQR",
# NBEA_HUMAN
"MASEKPGPGPGLEPQPVGLIAVGAAGGGGGGSGGGGTGGSGMGELRGASGSGSVMLPAGMINPSVPIRNIRMKFAVLIGLIQVGEVSNRDIVETVLNLLVGGEFDLEMNFIIQDAESITCMTELLEHCDVTCQAEIWSMFTAILRKSVRNLQTSTEVGLIEQVLLKMSAVDDMIADLLVDMLGVLASYSITVKELKLLFSMLRGESGIWPRHAVKLLSVLNQMPQRHGPDTFFNFPGCSAAAIALPPIAKWPYQNGFTLNTWFRMDPLNNINVDKDKPYLYCFRTSKGVGYSAHFVGNCLIVTSLKSKGKGFQHCVKYDFQPRKWYMISIVHIYNRWRNSEIRCYVNGQLVSYGDMAWHVNTNDSYDKCFLGSSETADANRVFCGQLGAVYVFSEALNPAQIFAIHQLGPGYKSTFKFKSESDIHLAEHHKQVLYDGKLASSIAFTYNAKATDAQLCLESSPKENASIFVHSPHALMLQDVKAIVTHSIHSAIHSIGGIQVLFPLFAQLDNRQLNDSQVETTVCATLLAFLVELLKSSVAMQEQMLGGKGFLVIGYLLEKSSRVHITRAVLEQFLSFAKYLDGLSHGAPLLKQLCDHILFNPAIWIHTPAKVQLSLYTYLSAEFIGTATIYTTIRRVGTVLQLMHTLKYYYWVINPADSSGITPKGLDGPRPSQKEIISLRAFMLLFLKQLILKDRGVKEDELQSILNYLLTMHEDENIHDVLQLLVALMSEHPASMIPAFDQRNGIRVIYKLLASKSESIWVQALKVLGYFLKHLGHKRKVEIMHTHSLFTLLGERLMLHTNTVTVTTYNTLYEILTEQVCTQVVHKPHPEPDSTVKIQNPMILKVVATLLKNSTPSAELMEVRRLFLSDMIKLFSNSRENRRCLLQCSVWQDWMFSLGYINPKNSEEQKITEMVYNIFRILLYHAIKYEWGGWRVWVDTLSIAHSKVTYEAHKEYLAKMYEEYQRQEEENIKKGKKGNVSTISGLSSQTTGAKGGMEIREIEDLSQSQSPESETDYPVSTDTRDLLMSTKVSDDILGNSDRPGSGVHVEVHDLLVDIKAEKVEATEVKLDDMDLSPETLVGGENGALVEVESLLDNVYSAAVEKLQNNVHGSVGIIKKNEEKDNGPLITLADEKEDLPNSSTSFLFDKIPKQEEKLLPELSSNHIIPNIQDTQVHLGVSDDLGLLAHMTGSVDLTCTSSIIEEKEFKIHTTSDGMSSISERDLASSTKGLEYAEMTATTLETESSSSKIVPNIDAGSIISDTERSDDGKESGKEIRKIQTTTTTQAVQGRSITQQDRDLRVDLGFRGMPMTEEQRRQFSPGPRTTMFRIPEFKWSPMHQRLLTDLLFALETDVHVWRSHSTKSVMDFVNSNENIIFVHNTIHLISQMVDNIIIACGGILPLLSAATSPTGSKTELENIEVTQGMSAETAVTFLSRLMAMVDVLVFASSLNFSEIEAEKNMSSGGLMRQCLRLVCCVAVRNCLECRQRQRDRGNKSSHGSSKPQEVPQSVTATAASKTPLENVPGNLSPIKDPDRLLQDVDINRLRAVVFRDVDDSKQAQFLALAVVYFISVLMVSKYRDILEPQRETTRTGSQPGRNIRQEINSPTSTVVVIPSIPHPSLNHGFLAKLIPEQSFGHSFYKETPAAFPDTIKEKETPTPGEDIQVESSIPHTDSGIGEEQVASILNGAELETSTGPDAMSELLSTLSSEVKKSQESLTENPSETLKPATSISSISQTKGINVKEILKSLVAAPVEIAECGPEPIPYPDPALKRETQAILPMQFHSFDRSVVVPVKKPPPGSLAVTTVGATTAGSGLPTGSTSNIFAATGATPKSMINTTGAVDSGSSSSSSSSSFVNGATSKNLPAVQTVAPMPEDSAENMSITAKLERALEKVAPLLREIFVDFAPFLSRTLLGSHGQELLIEGLVCMKSSTSVVELVMLLCSQEWQNSIQKNAGLAFIELINEGRLLCHAMKDHIVRVANEAEFILNRQRAEDVHKHAEFESQCAQYAADRREEEKMCDHLISAAKHRDHVTANQLKQKILNILTNKHGAWGAVSHSQLHDFWRLDYWEDDLRRRRRFVRNAFGSTHAEALLKAAIEYGTEEDVVKSKKTFRSQAIVNQNAETELMLEGDDDAVSLLQEKEIDNLAGPVVLSTPAQLIAPVVVAKGTLSITTTEIYFEVDEDDSAFKKIDTKVLAYTEGLHGKWMFSEIRAVFSRRYLLQNTALEVFMANRTSVMFNFPDQATVKKVVYSLPRVGVGTSYGLPQARRISLATPRQLYKSSNMTQRWQRREISNFEYLMFLNTIAGRTYNDLNQYPVFPWVLTNYESEELDLTLPGNFRDLSKPIGALNPKRAVFYAERYETWEDDQSPPYHYNTHYSTATSTLSWLVRIEPFTTFFLNANDGKFDHPDRTFSSVARSWRTSQRDTSDVKELIPEFYYLPEMFVNSNGYNLGVREDEVVVNDVDLPPWAKKPEDFVRINRMALESEFVSCQLHQWIDLIFGYKQRGPEAVRALNVFHYLTYEGSVNLDSITDPVLREAMEAQIQNFGQTPSQLLIEPHPPRSSAMHLCFLPQSPLMFKDQMQQDVIMVLKFPSNSPVTHVAANTLPHLTIPAVVTVTCSRLFAVNRWHNTVGLRGAPGYSLDQAHHLPIEMDPLIANNSGVNKRQITDLVDQSIQINAHCFVVTADNRYILICGFWDKSFRVYSTETGKLTQIVFGHWDVVTCLARSESYIGGDCYIVSGSRDATLLLWYWSGRHHIIGDNPNSSDYPAPRAVLTGHDHEVVCVSVCAELGLVISGAKEGPCLVHTITGDLLRALEGPENCLFPRLISVSSEGHCIIYYERGRFSNFSINGKLLAQMEINDSTRAILLSSDGQNLVTGGDNGVVEVWQACDFKQLYIYPGCDAGIRAMDLSHDQRTLITGMASGSIVAFNIDFNRWHYEHQNRY",
# NLGNY_HUMAN
"MLRPQGLLWLPLLFTSVCVMLNSNVLLWITALAIKFTLIDSQAQYPVVNTNYGKIQGLRTPLPSEILGPVEQYLGVPYASPPTGERRFQPPESPSSWTGIRNATQFSAVCPQHLDERFLLHDMLPIWFTTSLDTLMTYVQDQNEDCLYLNIYVPMEDDIHEQNSKKPVMVYIHGGSYMEGTGNMIDGSILASYGNVIVITINYRLGILGFLSTGDQAAKGNYGLLDQIQALRWIEENVGAFGGDPKRVTIFGSGAGASCVSLLTLSHYSEGLFQKAIIQSGTALSSWAVNYQPAKYTRILADKVGCNMLDTTDMVECLKNKNYKELIQQTITPATYHIAFGPVIDGDVIPDDPQILMEQGEFLNYDIMLGVNQGEGLKFVDGIVDNEDGVTPNDFDFSVSNFVDNLYGYPEGKDTLRETIKFMYTDWADKENPETRRKTLVALFTDHQWVAPAVATADLHAQYGSPTYFYAFYHHCQSEMKPSWADSAHGDEVPYVFGIPMIGPTELFSCNFSKNDVMLSAVVMTYWTNFAKTGDPNQPVPQDTKFIHTKPNRFEEVAWSKYNPKDQLYLHIGLKPRVRDHYRATKVAFWLELVPHLHNLNEIFQYVSTTTKVPPPDMTSFPYGTRRSPAKIWPTTKRPAITPANNPKHSKDPHKTGPEDTTVLIETKRDYSTELSVTIAVGASLLFLNILAFAALYYKKDKRRHETHRHPSPQRNTTNDITHIQNEEIMSLQMKQLEHDHECESLQAHDTLRLTCPPDYTLTLRRSPDDIPFMTPNTITMIPNTLMGMQPLHTFKTFSGGQNSTNLPHGHSTTRV",
# NLGN2_HUMAN
"MWLLALCLVGLAGAQRGGGGPGGGAPGGPGLGLGSLGEERFPVVNTAYGRVRGVRRELNNEILGPVVQFLGVPYATPPLGARRFQPPEAPASWPGVRNATTLPPACPQNLHGALPAIMLPVWFTDNLEAAATYVQNQSEDCLYLNLYVPTEDGPLTKKRDEATLNPPDTDIRDPGKKPVMLFLHGGSYMEGTGNMFDGSVLAAYGNVIVATLNYRLGVLGFLSTGDQAAKGNYGLLDQIQALRWLSENIAHFGGDPERITIFGSGAGASCVNLLILSHHSEGLFQKAIAQSGTAISSWSVNYQPLKYTRLLAAKVGCDREDSAEAVECLRRKPSRELVDQDVQPARYHIAFGPVVDGDVVPDDPEILMQQGEFLNYDMLIGVNQGEGLKFVEDSAESEDGVSASAFDFTVSNFVDNLYGYPEGKDVLRETIKFMYTDWADRDNGEMRRKTLLALFTDHQWVAPAVATAKLHADYQSPVYFYTFYHHCQAEGRPEWADAAHGDELPYVFGVPMVGATDLFPCNFSKNDVMLSAVVMTYWTNFAKTGDPNQPVPQDTKFIHTKPNRFEEVVWSKFNSKEKQYLHIGLKPRVRDNYRANKVAFWLELVPHLHNLHTELFTTTTRLPPYATRWPPRPPAGAPGTRRPPPPATLPPEPEPEPGPRAYDRFPGDSRDYSTELSVTVAVGASLLFLNILAFAALYYKRDRRQELRCRRLSPPGGSGSGVPGGGPLLPAAGRELPPEEELVSLQLKRGGGVGADPAEALRPACPPDYTLALRRAPDDVPLLAPGALTLLPSGLGPPPPPPPPSLHPFGPFPPPPPTATSHNNTLPHPHSTTRV",
# CADM4_HUMAN
"MGRARRFQWPLLLLWAAAAGPGAGQEVQTENVTVAEGGVAEITCRLHQYDGSIVVIQNPARQTLFFNGTRALKDERFQLEEFSPRRVRIRLSDARLEDEGGYFCQLYTEDTHHQIATLTVLVAPENPVVEVREQAVEGGEVELSCLVPRSRPAATLRWYRDRKELKGVSSSQENGKVWSVASTVRFRVDRKDDGGIIICEAQNQALPSGHSKQTQYVLDVQYSPTARIHASQAVVREGDTLVLTCAVTGNPRPNQIRWNRGNESLPERAEAVGETLTLPGLVSADNGTYTCEASNKHGHARALYVLVVYDPGAVVEAQTSVPYAIVGGILALLVFLIICVLVGMVWCSVRQKGSYLTHEASGLDEQGEAREAFLNGSDGHKRKEEFFI",
# VA0E2_HUMAN
"MTAHSFALPVIIFTTFWGLVGIAGPWFVPKGPNRGVIITMLVATAVCCYLFWLIAILAQLNPLFGPQLKNETIWYVRFLWE",
# SCAM5_HUMAN
"MAEKVNNFPPLPKFIPLKPCFYQDFEADIPPQHVSMTKRLYYLWMLNSVTLAVNLVGCLAWLIGGGGATNFGLAFLWLILFTPCSYVCWFRPIYKAFKTDSSFSFMAFFFTFMAQLVISIIQAVGIPGWGVCGWIATISFFGTNIGSAVVMLIPTVMFTVMAVFSFIALSMVHKFYRGSGGSFSKAQEEWTTGAWKNPHVQQAAQNAAMGAAQGAMNQPQTQYSATPNYTYSNEM",
# PRR7_HUMAN
"MVMSQGTYTFLTCFAGFWLIWGLIVLLCCFCSFLRRRLKRRQEERLREQNLRALELEPLELEGSLAGSPPGLAPPQPPPHRSRLEAPAHAHSHPHVHVHPLLHHGPAQPHAHAHPHPHHHALPHPPPTHLSVPPRPWSYPRQAESDMSKPPCYEEAVLMAEPPPPYSEVLTDTRGLYRKIVTPFLSRRDSAEKQEQPPPSYKPLFLDRGYTSALHLPSAPRPAPPCPALCLQADRGRRVFPSWTDSELSSREPLEHGAWRLPVSIPLFGRTTAV",
# PUM2_HUMAN
"MNHDFQALALESRGMGELLPTKKFWEPDDSTKDGQKGIFLGDDEWRETAWGASHHSMSQPIMVQRRSGQGFHGNSEVNAILSPRSESGGLGVSMVEYVLSSSPADKLDSRFRKGNFGTRDAETDGPEKGDQKGKASPFEEDQNRDLKQGDDDDSKINGRGLPNGMDADCKDFNRTPGSRQASPTEVVERLGPNTNPSEGLGPLPNPTANKPLVEEFSNPETQNLDAMEQVGLESLQFDYPGNQVPMDSSGATVGLFDYNSQQQLFQRTNALTVQQLTAAQQQQYALAAAQQPHIAGVFSAGLAPAAFVPNPYIISAAPPGTDPYTAAGLAAAATLAGPAVVPPQYYGVPWGVYPANLFQQQAAAAANNTASQQAASQAQPGQQQVLRAGAGQRPLTPNQGQQGQQAESLAAAAAANPTLAFGQGLATGMPGYQVLAPTAYYDQTGALVVGPGARTGLGAPVRLMAPTPVLISSAAAQAAAAAAAGGTASSLTGSTNGLFRPIGTQPPQQQQQQPSTNLQSNSFYGSSSLTNSSQSSSLFSHGPGQPGSTSLGFGSGNSLGAAIGSALSGFGSSVGSSASSSATRRESLSTSSDLYKRSSSSLAPIGQPFYNSLGFSSSPSPIGMPLPSQTPGHSLTPPPSLSSHGSSSSLHLGGLTNGSGRYISAAPGAEAKYRSASSTSSLFSSSSQLFPPSRLRYNRSDIMPSGRSRLLEDFRNNRFPNLQLRDLIGHIVEFSQDQHGSRFIQQKLERATPAERQMVFNEILQAAYQLMTDVFGNYVIQKFFEFGSLDQKLALATRIRGHVLPLALQMYGCRVIQKALESISSDQQVISEMVKELDGHVLKCVKDQNGNHVVQKCIECVQPQSLQFIIDAFKGQVFVLSTHPYGCRVIQRILEHCTAEQTLPILEELHQHTEQLVQDQYGNYVIQHVLEHGRPEDKSKIVSEIRGKVLALSQHKFASNVVEKCVTHASRAERALLIDEVCCQNDGPHSALYTMMKDQYANYVVQKMIDMAEPAQRKIIMHKIRPHITTLRKYTYGKHILAKLEKYYLKNSPDLGPIGGPPNGML",
# LNX1_HUMAN
"MNQPESANDPEPLCAVCGQAHSLEENHFYSYPEEVDDDLICHICLQALLDPLDTPCGHTYCTLCLTNFLVEKDFCPMDRKPLVLQHCKKSSILVNKLLNKLLVTCPFREHCTQVLQRCDLEHHFQTSCKGASHYGLTKDRKRRSQDGCPDGCASLTATAPSPEVSAAATISLMTDEPGLDNPAYVSSAEDGQPAISPVDSGRSNRTRARPFERSTIRSRSFKKINRALSVLRRTKSGSAVANHADQGRENSENTTAPEVFPRLYHLIPDGEITSIKINRVDPSESLSIRLVGGSETPLVHIIIQHIYRDGVIARDGRLLPGDIILKVNGMDISNVPHNYAVRLLRQPCQVLWLTVMREQKFRSRNNGQAPDAYRPRDDSFHVILNKSSPEEQLGIKLVRKVDEPGVFIFNVLDGGVAYRHGQLEENDRVLAINGHDLRYGSPESAAHLIQASERRVHLVVSRQVRQRSPDIFQEAGWNSNGSWSPGPGERSNTPKPLHPTITCHEKVVNIQKDPGESLGMTVAGGASHREWDLPIYVISVEPGGVISRDGRIKTGDILLNVDGVELTEVSRSEAVALLKRTSSSIVLKALEVKEYEPQEDCSSPAALDSNHNMAPPSDWSPSWVMWLELPRCLYNCKDIVLRRNTAGSLGFCIVGGYEEYNGNKPFFIKSIVEGTPAYNDGRIRCGDILLAVNGRSTSGMIHACLARLLKELKGRITLTIVSWPGTFL",
# CNIH3_HUMAN
"MAFTFAAFCYMLSLVLCAALIFFAIWHIIAFDELRTDFKSPIDQCNPVHARERLRNIERICFLLRKLVLPEYSIHSLFCIMFLCAQEWLTLGLNVPLLFYHFWRYFHCPADSSELAYDPPVVMNADTLSYCQKEAWCKLAFYLLSFFYYLYCMIYTLVSS",
# FA81A_HUMAN
"MENMHLRRVRTMPRHSQSLTMAPYSSVSLVEQLEDRILCHEKTTAALVEHAFRIKDDIVNSLQKMQNKGGGDRLARLFLEEHIRNITAIVKQLNRDIEVLQEQIRARDNISYGTNSALKTLEMRQLSGLGDLRGRVARCDASIARLSAEHKTTYEGLQHLNKEQQAAKLILETKIKDAEGQISQLLNRVDLSISEQSTKLKMSHRDSNHQLQLLDTKFKGTVEELSNQILSARSWLQQEQERIEKELLQKIDQLSLIVKENSGASERDMEKKLSQMSARLDKIEEGQKKTFDGQRTRQEEEKMHGRITKLELQMNQNIKEMKAEVNAGFTAVYESIGSLRQVLEAKMKLDRDQLQKQIQLMQKPETPM",
# SYNPR_HUMAN
"MCMVIFAPLFAIFAFATCGGYSGGLRLSVDCVNKTESNLSIDIAFAYPFRLHQVTFEVPTCEGKERQKLALIGDSSSSAEFFVTVAVFAFLYSLAATVVYIFFQNKYRENNRGPLIDFIVTVVFSFLWLVGSSAWAKGLSDVKVATDPKEVLLLMSACKQPSNKCMAIHSPVMSSLNTSVVFGFLNFILWAGNIWFVFKETGWHSSGQRYLSDPMEKHSSSYNQGGYNQDSYGSSSGYSQQASLGPTSDEFGQQPTGPTSFTNQI",
# TM163_HUMAN
"MEPAAGIQRRSSQGPTVPPPPRGHAPPAAAPGPAPLSSPVREPPQLEEERQVRISESGQFSDGLEDRGLLESSTRLKPHEAQNYRKKALWVSWFSIIVTLALAVAAFTVSVMRYSASAFGFAFDAILDVLSSAIVLWRYSNAAAVHSAHREYIACVILGVIFLLSSICIVVKAIHDLSTRLLPEVDDFLFSVSILSGILCSILAVLKFMLGKVLTSRALITDGFNSLVGGVMGFSILLSAEVFKHDSAVWYLDGSIGVLIGLTIFAYGVKLLIDMVPRVRQTRHYEMFE",
# CPT1C_HUMAN
"MAEAHQAVGFRPSLTSDGAEVELSAPVLQEIYLSGLRSWKRHLSRFWNDFLTGVFPASPLSWLFLFSAIQLAWFLQLDPSLGLMEKIKELLPDWGGQHHGLRGVLAAALFASCLWGALIFTLHVALRLLLSYHGWLLEPHGAMSSPTKTWLALVRIFSGRHPMLFSYQRSLPRQPVPSVQDTVRKYLESVRPILSDEDFDWTAVLAQEFLRLQASLLQWYLRLKSWWASNYVSDWWEEFVYLRSRNPLMVNSNYYMMDFLYVTPTPLQAARAGNAVHALLLYRHRLNRQEIPPTLLMGMRPLCSAQYEKIFNTTRIPGVQKDYIRHLHDSQHVAVFHRGRFFRMGTHSRNSLLSPRALEQQFQRILDDPSPACPHEEHLAALTAAPRGTWAQVRTSLKTQAAEALEAVEGAAFFVSLDAEPAGLTREDPAASLDAYAHALLAGRGHDRWFDKSFTLIVFSNGKLGLSVEHSWADCPISGHMWEFTLATECFQLGYSTDGHCKGHPDPTLPQPQRLQWDLPDQIHSSISLALRGAKILSENVDCHVVPFSLFGKSFIRRCHLSSDSFIQIALQLAHFRDRGQFCLTYESAMTRLFLEGRTETVRSCTREACNFVRAMEDKEKTDPQCLALFRVAVDKHQALLKAAMSGQGVDRHLFALYIVSRFLHLQSPFLTQVHSEQWQLSTSQIPVQQMHLFDVHNYPDYVSSGGGFGPADDHGYGVSYIFMGDGMITFHISSKKSSTKTDSHRLGQHIEDALLDVASLFQAGQHFKRRFRGSGKENSRHRCGFLSRQTGASKASMTSTDF",
# NMD3A_HUMAN
"MRRLSLWWLLSRVCLLLPPPCALVLAGVPSSSSHPQPCQILKRIGHAVRVGAVHLQPWTTAPRAASRAPDDSRAGAQRDEPEPGTRRSPAPSPGARWLGSTLHGRGPPGSRKPGEGARAEALWPRDALLFAVDNLNRVEGLLPYNLSLEVVMAIEAGLGDLPLLPFSSPSSPWSSDPFSFLQSVCHTVVVQGVSALLAFPQSQGEMMELDLVSLVLHIPVISIVRHEFPRESQNPLHLQLSLENSLSSDADVTVSILTMNNWYNFSLLLCQEDWNITDFLLLTQNNSKFHLGSIINITANLPSTQDLLSFLQIQLESIKNSTPTVVMFGCDMESIRRIFEITTQFGVMPPELRWVLGDSQNVEELRTEGLPLGLIAHGKTTQSVFEHYVQDAMELVARAVATATMIQPELALIPSTMNCMEVETTNLTSGQYLSRFLANTTFRGLSGSIRVKGSTIVSSENNFFIWNLQHDPMGKPMWTRLGSWQGGKIVMDYGIWPEQAQRHKTHFQHPSKLHLRVVTLIEHPFVFTREVDDEGLCPAGQLCLDPMTNDSSTLDSLFSSLHSSNDTVPIKFKKCCYGYCIDLLEKIAEDMNFDFDLYIVGDGKYGAWKNGHWTGLVGDLLRGTAHMAVTSFSINTARSQVIDFTSPFFSTSLGILVRTRDTAAPIGAFMWPLHWTMWLGIFVALHITAVFLTLYEWKSPFGLTPKGRNRSKVFSFSSALNICYALLFGRTVAIKPPKCWTGRFLMNLWAIFCMFCLSTYTANLAAVMVGEKIYEELSGIHDPKLHHPSQGFRFGTVRESSAEDYVRQSFPEMHEYMRRYNVPATPDGVEYLKNDPEKLDAFIMDKALLDYEVSIDADCKLLTVGKPFAIEGYGIGLPPNSPLTANISELISQYKSHGFMDMLHDKWYRVVPCGKRSFAVTETLQMGIKHFSGLFVLLCIGFGLSILTTIGEHIVYRLLLPRIKNKSKLQYWLHTSQRLHRAINTSFIEEKQQHFKTKRVEKRSNVGPRQLTVWNTSNLSHDNRRKYIFSDEEGQNQLGIRIHQDIPLPPRRRELPALRTTNGKADSLNVSRNSVMQELSELEKQIQVIRQELQLAVSRKTELEEYQRTSRTCES",
# BRSK1_HUMAN
"MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTGLVKLGVHCITGQKVAIKIVNREKLSESVLMKVEREIAILKLIEHPHVLKLHDVYENKKYLYLVLEHVSGGELFDYLVKKGRLTPKEARKFFRQIVSALDFCHSYSICHRDLKPENLLLDEKNNIRIADFGMASLQVGDSLLETSCGSPHYACPEVIKGEKYDGRRADMWSCGVILFALLVGALPFDDDNLRQLLEKVKRGVFHMPHFIPPDCQSLLRGMIEVEPEKRLSLEQIQKHPWYLGGKHEPDPCLEPAPGRRVAMRSLPSNGELDPDVLESMASLGCFRDRERLHRELRSEEENQEKMIYYLLLDRKERYPSCEDQDLPPRNDVDPPRKRVDSPMLSRHGKRRPERKSMEVLSITDAGGGGSPVPTRRALEMAQHSQRSRSVSGASTGLSSSPLSSPRSPVFSFSPEPGAGDEARGGGSPTSKTQTLPSRGPRGGGAGEQPPPPSARSTPLPGPPGSPRSSGGTPLHSPLHTPRASPTGTPGTTPPPSPGGGVGGAAWRSRLNSIRNSFLGSPRFHRRKMQVPTAEEMSSLTPESSPELAKRSWFGNFISLDKEEQIFLVLKDKPLSSIKADIVHAFLSIPSLSHSVLSQTSFRAEYKASGGPSVFQKPVRFQVDISSSEGPEPSPRRDGSGGGGIYSVTFTLISGPSRRFKRVVETIQAQLLSTHDQPSVQALADEKNGAQTRPAGAPPRSLQPPPGRPDPELSSSPRRGPPKDKKLLATNGTPLP",
# NETO1_HUMAN
"MIHGRSVLHIVASLIILHLSGATKKGTEKQTTSETQKSVQCGTWTKHAEGGIFTSPNYPSKYPPDRECIYIIEAAPRQCIELYFDEKYSIEPSWECKFDHIEVRDGPFGFSPIIGRFCGQQNPPVIKSSGRFLWIKFFADGELESMGFSARYNFTPDPDFKDLGALKPLPACEFEMGGSEGIVESIQIMKEGKATASEAVDCKWYIRAPPRSKIYLRFLDYEMQNSNECKRNFVAVYDGSSSVEDLKAKFCSTVANDVMLRTGLGVIRMWADEGSRNSRFQMLFTSFQEPPCEGNTFFCHSNMCINNTLVCNGLQNCVYPWDENHCKEKRKTSLLDQLTNTSGTVIGVTSCIVIILIIISVIVQIKQPRKKYVQRKSDFDQTVFQEVFEPPHYELCTLRGTGATADFADVADDFENYHKLRRSSSKCIHDHHCGSQLSSTKGSRSNLSTRDASILTEMPTQPGKPLIPPMNRRNILVMKHSYSQDAADACDIDEIEEVPTTSHRLSRHDKAVQRFCLIGSLSKHESEYNTTRV",
# DMXL2_HUMAN
"MHLHQVLTGAVNPGDNCYSVGSVGDVPFTAYGSGCDIVILANDFECVQIIPGAKHGNIQVSCVECSNQQGRIAASYGNAVCIFEPLGINSHKRNCQLKCQWLKTGQFFLSSVTYNLAWDPQDNRLLTATDSIQLWAPPGDDILEEEEEIDNTVPPVLNDWKCVWQCKTSVSVHLMEWSPDGEYFATAGKDDCLLKVWYPMTGWKSSIIPQDHHEVKRRQSSTQFSFVYLAHPRAVTGFSWRKTSKYMPRGSVCNVLLTSCHDGVCRLWAETLLPEDCLLGEQICETTTSSIASSLSHAGRHKDRIQHALETIHHLKNLRKGQRRSSVLVTHAELMPDQTAMHEVQRHISHHANALCHFHIAASINPATDIPNVLVGTAFNVDDGNGGFVVHWLNNKEFHFTSSTEVFMHQLRKLSDKQVDHENDDADREDEEHSQEDRERGLHMKLDHDLSLDRESEAGTGSSEHEDGEREGSPRTYSRLSVPMPLPTVLLDRKIETLLTEWNKNPDMLFTIHPVDGTFLVWHVKYLDEYNPGIFRQVQVSFSSRIPVAFPSGDASSLSKNIMMYACINATKDSHHTLLHQEGMSVGSPHGSQPHSRSHSTHMNILAPTVMMISKHIDGSLNQWAVTFADKSAFTTVLTVSHKFRYCGHRFHLNDLACHSVLPLLLTSSHHNALLTPELDCQWDSDNKLSRLMDPVKHIKGSSKQPLRNAATRTFHDPNAIYSELILWRVDPIGPLSYTGGVSELARINSLHTSAFSNVAWLPTLIPSYCLGTYCNSASACFVASDGKNLRLYQAVVDARKLLDELSDPESSKLIGEVFNIVSQQSTARPGCIIELDAITNQCGSNTQLLHVFQEDFIIGYKPHKEDMEKKETEIFFQPSQGYRPPPFSEKFFLVVIEKDSNNNSILHMWHLHLKSVQACLAKASEGASSESLLSVPGQKNVDSSPETSPSVSPMPHSSSIANLQTASKLILSSRLVYSQPLDLPESVEVIRATPSAGHLSSSSIYPVCLAPYLVVTTCSDNKVRFWKCCMEANPECNKSDEKEIYHWKRWPLMNDEGEDNSSTVSIVGRPVAVSCSYTGRLAVAYKQPIHHNGFVSKEFSMHVCIFECESTGGSEWVLEQTIHLDDLVKVGSVLDSRVSVDSNLFVYSKSDALLSKDRYLIPNIKHLVHLDWVSKEDGSHILTVGVGANIFMYGRLSGIVTEQTNSKDGVAVITLPLGGSIKQGVKSRWVLLRSIDLVSSVDGTPSLPVSLSWVRDGILVVGMDCEMHVYAQWKHAVKFGDTEADSSNAEEAAMQDHSTFKSNMLARKSVVEGTAISDDVFCSPTVIQDGGLFEAAHVLSPTLPQYHPTQLLELMDLGKVRRAKAILSHLVKCIAGEVAIVRDPDAGEGTKRHLSRTISVSGSTAKETVTVGKDGTRDYTEIDSIPPLPLYALLAADQDTSYRISEESTKIPQSYEDQTVSQPEDQYSELFQIQDIPTDDIDLEPEKRENKSKVINLSQYGPAYFGQEHARVLSSHLMHSSLPGLTRLEQMFLVALADTVATTSTELDESRDKSCSGRDTLDECGLRYLLAMRLHTCLLTSLPPLYRVQLLHQGVSTCHFAWAFHSEAEEELINMIPAIQRGDPQWSELRAMGIGWWVRNINTLRRCIEKVAKASFQRNNDALDAALFYLSMKKKAVVWGLFRSQHDEKMTTFFSHNFNEDRWRKAALKNAFSLLGKQRFEQSAAFFLLAGSLKDAIEVCLEKMEDIQLAMVIARLYESEFETSSTYISILNQKILGCQKDGSGFSCKRLHPDPFLRSLAYWVMKDYTRALDTLLEQTPKEDDEHQVIIKSCNPVAFSFYNYLRTHPLLIRRNLASPEGTLATLGLKTEKNFVDKINLIERKLFFTTANAHFKVGCPVLALEVLSKIPKVTKTSALSAKKDQPDFISHRMDDVPSHSKALSDGNGSSGIEWSNVTSSQYDWSQPIVKVDEEPLNLDWGEDHDSALDEEEDDAVGLVMKSTDAREKDKQSDQKASDPNMLLTPQEEDDPEGDTEVDVIAEQLKFRACLKILMTELRTLATGYEVDGGKLRFQLYNWLEKEIAALHEICNHESVIKEYSSKTYSKVESDLLDQEEMVDKPDIGSYERHQIERRRLQAKREHAERRKSWLQKNQDLLRVFLSYCSLHGAQGGGLASVRMELKFLLQESQQETTVKQLQSPLPLPTTLPLLSASIASTKTVIANPVLYLNNHIHDILYTIVQMKTPPHPSIEDVKVHTLHSLAASLSASIYQALCDSHSYSQTEGNQFTGMAYQGLLLSDRRRLRTESIEEHATPNSSPAQWPGVSSLINLLSSAQDEDQPKLNILLCEAVVAVYLSLLIHALATNSSSELFRLAAHPLNNRMWAAVFGGGVKLVVKPRRQSENISAPPVLSEDIDKHRRRFNMRMLVPGRPVKDATPPPVPAERPSYKEKFIPPELSMWDYFVAKPFLPLSDSGVIYDSDESIHSDEEDDAFFSDTQIQEHQDPNSYSWALLHLTMVKLALHNVKNFFPIAGLEFSELPVTSPLGIAVIKNLENWEQILQEKMDQFEGPPPNYINTYPTDLSVGAGPAILRNKAMLEPENTPFKSRDSSAFPVKRLWHFLVKQEVLQETFIRYIFTKKRKQSEVEADLGYPGGKAKVIHKESDMIMAFSVNKANCNEIVLASTHDVQELDVTSLLACQSYIWIGEEYDRESKSSDDVDYRGSTTTLYQPSATSYSASQVHPPSSLPWLGTGQTSTGASVLMKRNLHNVKRMTSHPVHQYYLTGAQDGSVRMFEWTRPQQLVCFRQAGNARVTRLYFNSQGNKCGVADGEGFLSIWQVNQTASNPKPYMSWQCHSKATSDFAFITSSSLVATSGHSNDNRNVCLWDTLISPGNSLIHGFTCHDHGATVLQYAPKQQLLISGGRKGHVCIFDIRQRQLIHTFQAHDSAIKALALDPYEEYFTTGSAEGNIKVWRLTGHGLIHSFKSEHAKQSIFRNIGAGVMQIDIIQGNRLFSCGADGTLKTRVLPNAFNIPNRILDIL",
# DLG5_HUMAN
"MEPQRRELLAQCQQSLAQAMTEVEAVLGLLEAAGALSPGERRQLDEEAGGAKAELLLKLLLAKERDHFQDLRAALEKTQPHLLPILYLNGVVGPPQPAEGAGSTYSVLSTMPSDSESSSSLSSVGTTGKAPSPPPLLTDQQVNEKVENLSIQLRLMTRERNELRKRLAFATHGTAFDKRPYHRLNPDYERLKIQCVRAMSDLQSLQNQHTNALKRCEEVAKETDFYHTLHSRLLSDQTRLKDDVDMLRRENGQLLRERNLLQQSWEDMKRLHEEDQKEIGDLRAQQQQVLKHNGSSEILNKLYDTAMDKLEVVKKDYDALRKRYSEKVAIHNADLSRLEQLGEENQRLLKQTEMLTQQRDTAIQLQHQCALSLRRFEAIHHELNKATAQNKDLQWEMELLQSELTELRTTQVKTAKESEKYREERDAVYSEYKLIMSERDQVISELDKLQTEVELAESKLKSSTSEKKAANEEMEALRQIKDTVTMDAGRANKEVEILRKQCKALCQELKEALQEADVAKCRRDWAFQERDKIVAERDSIRTLCDNLRRERDRAVSELAEALRSLDDTRKQKNDVSRELKELKEQMESQLEKEARFRQLMAHSSHDSAIDTDSMEWETEVVEFERETEDIDLKALGFDMAEGVNEPCFPGDCGIFVTKVDKGSIADGRLRVNDWLLRINDVDLINKDKKQAIKALLNGEGAINMVVRRRKSLGGKVVTPLHINLSGQKDSGISLENGVYAAAVLPGSPAAKEGSLAVGDRIVAINGIALDNKSLNECESLLRSCQDSLTLSLLKVFPQSSSWSGQNIFENIKDSDKMLSFRAHGPEVQAHNKRNLIQHNNSTQTDIFYTDRLEDRKEPGPPGGSSSFLHKPFPGGPLQVCPQACPSASERSLSSFRSDASGDRGFGLVDVRGRRPLLPFETEVGPCGVGEASLDKADSEGSNSGGTWPKAMLSSTAVPEKLSVYKKPKQRKSIFDPNTFKRPQTPPKIDYLLPGPGPAHSPQPSKRAGPLTPPKPPRRSDSIKFQHRLETSSESEATLVGSSPSTSPPSALPPDVDPGEPMHASPPRKARVRIASSYYPEGDGDSSHLPAKKSCDEDLTSQKVDELGQKRRRPKSAPSFRPKLAPVVIPAQFLEEQKCVPASGELSPELQEWAPYSPGHSSRHSNPPLYPSRPSVGTVPRSLTPSTTVSSILRNPIYTVRSHRVGPCSSPPAARDAGPQGLHPSVQHQGRLSLDLSHRTCSDYSEMRATHGSNSLPSSARLGSSSNLQFKAERIKIPSTPRYPRSVVGSERGSVSHSECSTPPQSPLNIDTLSSCSQSQTSASTLPRIAVNPASLGERRKDRPYVEEPRHVKVQKGSEPLGISIVSGEKGGIYVSKVTVGSIAHQAGLEYGDQLLEFNGINLRSATEQQARLIIGQQCDTITILAQYNPHVHQLSSHSRSSSHLDPAGTHSTLQGSGTTTPEHPSVIDPLMEQDEGPSTPPAKQSSSRIAGDANKKTLEPRVVFIKKSQLELGVHLCGGNLHGVFVAEVEDDSPAKGPDGLVPGDLILEYGSLDVRNKTVEEVYVEMLKPRDGVRLKVQYRPEEFTKAKGLPGDSFYIRALYDRLADVEQELSFKKDDILYVDDTLPQGTFGSWMAWQLDENAQKIQRGQIPSKYVMDQEFSRRLSMSEVKDDNSATKTLSAAARRSFFRRKHKHKRSGSKDGKDLLALDAFSSDSIPLFEDSVSLAYQRVQKVDCTALRPVLILGPLLDVVKEMLVNEAPGKFCRCPLEVMKASQQAIERGVKDCLFVDYKRRSGHFDVTTVASIKEITEKNRHCLLDIAPHAIERLHHMHIYPIVIFIHYKSAKHIKEQRDPIYLRDKVTQRHSKEQFEAAQKLEQEYSRYFTGVIQGGALSSICTQILAMVNQEQNKVLWIPACPL",
# GP151_HUMAN
"MLAAAFADSNSSSMNVSFAHLHFAGGYLPSDSQDWRTIIPALLVAVCLVGFVGNLCVIGILLHNAWKGKPSMIHSLILNLSLADLSLLLFSAPIRATAYSKSVWDLGWFVCKSSDWFIHTCMAAKSLTIVVVAKVCFMYASDPAKQVSIHNYTIWSVLVAIWTVASLLPLPEWFFSTIRHHEGVEMCLVDVPAVAEEFMSMFGKLYPLLAFGLPLFFASFYFWRAYDQCKKRGTKTQNLRNQIRSKQVTVMLLSIAIISALLWLPEWVAWLWVWHLKAAGPAPPQGFIALSQVLMFSISSANPLIFLVMSEEFREGLKGVWKWMITKKPPTVSESQETPAGNSEGLPDKVPSPESPASIPEKEKPSSPSSGKGKTEKAEIPILPDVEQFWHERDTVPSVQDNDPIPWEHEDQETGEGVK",
# PRUN2_HUMAN
"MEEFLQRAKSKLNRSKRLEKVHVVIGPKSCDLDSLISTFTYAYFLDKVSPPGVLCLPVLNIPRTEFNYFTETRFILEELNISESFHIFRDEINLHQLNDEGKLSITLVGSSVLASEDKTLESAVVKVINPVEQSDANVEFRESSSSLVLKEILQEAPELITEQLAHRLRGSILFKWMTMESEKISEKQEEILSILEEKFPNLPPREDIINVLQETQFSAQGLSIEQTMLKDLKELSDGEIKVAISTVSMNLENCLFHSNITSDLKAFTDKFGFDVLILFSSYLSEEQQPRRQIAVYSENMELCSQICCELEECQNPCLELEPFDCGCDEILVYQQEDPSVTCDQVVLVVKEVINRRCPEMVSNSRTSSTEAVAGSAPLSQGSSGIMELYGSDIEPQPSSVNFIENPPDLNDSNQAQVDANVDLVSPDSGLATIRSSRSSKESSVFLSDDSPVGEGAGPHHTLLPGLDSYSPIPEGAVAEEHAWSGEHGEHFDLFNFDPAPMASGQSQQSSHSADYSPADDFFPNSDLSEGQLPAGPEGLDGMGTNMSNYSSSSLLSGAGKDSLVEHDEEFVQRQDSPRDNSERNLSLTDFVGDESPSPERLKNTGKRIPPTPMNSLVESSPSTEEPASLYTEDMTQKATDTGHMGPPQTHARCSSWWGGLEIDSKNIADAWSSSEQESVFQSPESWKEHKPSSIDRRASDSVFQPKSLEFTKSGPWESEFGQPELGSNDIQDKNEESLPFQNLPMEKSPLPNTSPQGTNHLIEDFASLWHSGRSPTAMPEPWGNPTDDGEPAAVAPFPAWSAFGKEDHDEALKNTWNLHPTSSKTPSVRDPNEWAMAKSGFAFSSSELLDNSPSEINNEAAPEIWGKKNNDSRDHIFAPGNPSSDLDHTWTNSKPPKEDQNGLVDPKTRGKVYEKVDSWNLFEENMKKGGSDVLVPWEDSFLSYKCSDYSASNLGEDSVPSPLDTNYSTSDSYTSPTFAGDEKETEHKPFAKEEGFESKDGNSTAEETDIPPQSLQQSSRNRISSGPGNLDMWASPHTDNSSEINTTHNLDENELKTEHTDGKNISMEDDVGESSQSSYDDPSMMQLYNETNRQLTLLHSSTNSRQTAPDSLDLWNRVILEDTQSTATISDMDNDLDWDDCSGGAAIPSDGQTEGYMAEGSEPETRFTVRQLEPWGLEYQEANQVDWELPASDEHTKDSAPSEHHTLNEKSGQLIANSIWDSVMRDKDMSSFMLPGSSHITDSEQRELPPEIPSHSANVKDTHSPDAPAASGTSESEALISHLDKQDTERETLQSDAASLATRLENPGYFPHPDPWKGHGDGQSESEKEAQGATDRGHLDEEEVIASGVENASGISEKGQSDQELSSLVASEHQEICIKSGKISSLAVTFSPQTEEPEEVLEYEEGSYNLDSRDVQTGMSADNLQPKDTHEKHLMSQRNSGETTETSDGMNFTKYVSVPEKDLEKTEECNFLEPENVGGGPPHRVPRSLDFGDVPIDSDVHVSSTCSEITKNLDVKGSENSLPGAGSSGNFDRDTISSEYTHSSASSPELNDSSVALSSWGQQPSSGYQEENQGNWSEQNHQESELITTDGQVEIVTKVKDLEKNRINEFEKSFDRKTPTFLEIWNDSVDGDSFSSLSSPETGKYSEHSGTHQESNLIASYQEKNEHDISATVQPEDARVISTSSGSDDDSVGGEESIEEEIQVANCHVAEDESRAWDSLNESNKFLVTADPKSENIYDYLDSSEPAENENKSNPFCDNQQSSPDPWTFSPLTETEMQITAVEKEKRSSPETGTTGDVAWQISPKASFPKNEDNSQLEMLGFSADSTEWWKASPQEGRLIESPFERELSDSSGVLEINSSVHQNASPWGVPVQGDIEPVETHYTNPFSDNHQSPFLEGNGKNSHEQLWNIQPRQPDPDADKFSQLVKLDQIKEKDSREQTFVSAAGDELTPETPTQEQCQDTMLPVCDHPDTAFTHAEENSCVTSNVSTNEGQETNQWEQEKSYLGEMTNSSIATENFPAVSSPTQLIMKPGSEWDGSTPSEDSRGTFVPDILHGNFQEGGQLASAAPDLWIDAKKPFSLKADGENPDILTHCEHDSNSQASDSPDICHDSEAKQETEKHLSACMGPEVESSELCLTEPEIDEEPIYEPGREFVPSNAELDSENATVLPPIGYQADIKGSSQPASHKGSPEPSEINGDNSTGLQVSEKGASPDMAPILEPVDRRIPRIENVATSIFVTHQEPTPEGDGSWISDSFSPESQPGARALFDGDPHLSTENPALVPDALLASDTCLDISEAAFDHSFSDASGLNTSTGTIDDMSKLTLSEGHPETPVDGDLGKQDICSSEASWGDFEYDVMGQNIDEDLLREPEHFLYGGDPPLEEDSLKQSLAPYTPPFDLSYLTEPAQSAETIEEAGSPEDESLGCRAAEIVLSALPDRRSEGNQAETKNRLPGSQLAVLHIREDPESVYLPVGAGSNILSPSNVDWEVETDNSDLPAGGDIGPPNGASKEISELEEEKTIPTKEPEQIKSEYKEERCTEKNEDRHALHMDYILVNREENSHSKPETCEERESIAELELYVGSKETGLQGTQLASFPDTCQPASLNERKGLSAEKMSSKSDTRSSFESPAQDQSWMFLGHSEVGDPSLDARDSGPGWSGKTVEPFSELGLGEGPQLQILEEMKPLESLALEEASGPVSQSQKSKSRGRAGPDAVTLQAVTHDNEWEMLSPQPVQKNMIPDTEMEEETEFLELGTRISRPNGLLSEDVGMDIPFEEGVLSPSAADMRPEPPNSLDLNDTHPRRIKLTAPNINLSLDQSEGSILSDDNLDSPDEIDINVDELDTPDEADSFEYTGHDPTANKDSGQESESIPEYTAEEEREDNRLWRTVVIGEQEQRIDMKVIEPYRRVISHGGYYGDGLNAIIVFAACFLPDSSRADYHYVMENLFLYVISTLELMVAEDYMIVYLNGATPRRRMPGLGWMKKCYQMIDRRLRKNLKSFIIVHPSWFIRTILAVTRPFISSKFSSKIKYVNSLSELSGLIPMDCIHIPESIIKLDEELREASEAAKTSCLYNDPEMSSMEKDIDLKLKEKP",
# RN138_HUMAN
"MAEDLSAATSYTEDDFYCPVCQEVLKTPVRTTACQHVFCRKCFLTAMRESGAHCPLCRGNVTRRERACPERALDLENIMRKFSGSCRCCAKQIKFYRMRHHYKSCKKYQDEYGVSSIIPNFQISQDSVGNSNRSETSTSDNTETYQENTSSSGHPTFKCPLCQESNFTRQRLLDHCNSNHLFQIVPVTCPICVSLPWGDPSQITRNFVSHLNQRHQFDYGEFVNLQLDEETQYQTAVEESFQVNI",
# CPLX3_HUMAN
"MAFMVKTMVGGQLKNLTGSLGGGEDKGDGDKSAAEAQGMSREEYEEYQKQLVEEKMERDAQFTQRKAERATLRSHFRDKYRLPKNETDESQIQMAGGDVELPRELAKMIEEDTEEEEEKASVLGQLASLPGLNLGSLKDKAQATLGDLKQSAEKCHVM",
# HNRLL_HUMAN
"MSSSSSSPRETYEEDREYESQAKRLKTEEGEIDYSAEEGENRREATPRGGGDGGGGGRSFSQPEAGGSHHKVSVSPVVHVRGLCESVVEADLVEALEKFGTICYVMMMPFKRQALVEFENIDSAKECVTFAADEPVYIAGQQAFFNYSTSKRITRPGNTDDPSGGNKVLLLSIQNPLYPITVDVLYTVCNPVGKVQRIVIFKRNGIQAMVEFESVLCAQKAKAALNGADIYAGCCTLKIEYARPTRLNVIRNDNDSWDYTKPYLGRRDRGKGRQRQAILGEHPSSFRHDGYGSHGPLLPLPSRYRMGSRDTPELVAYPLPQASSSYMHGGNPSGSVVMVSGLHQLKMNCSRVFNLFCLYGNIEKVKFMKTIPGTALVEMGDEYAVERAVTHLNNVKLFGKRLNVCVSKQHSVVPSQIFELEDGTSSYKDFAMSKNNRFTSAGQASKNIIQPPSCVLHYYNVPLCVTEETFTKLCNDHEVLTFIKYKVFDAKPSAKTLSGLLEWECKTDAVEALTALNHYQIRVPNGSNPYTLKLCFSTSSHL",
# CSKI1_HUMAN
"MGKEQELVQAVKAEDVGTAQRLLQRPRPGKAKLLGSTKKINVNFQDPDGFSALHHAALNGNTELISLLLEAQAAVDIKDNKGMRPLHYAAWQGRKEPMKLVLKAGSAVNIPSDEGHIPLHLAAQHGHYDVSEMLLQHQSNPCMVDNSGKTPLDLACEFGRVGVVQLLLSSNMCAALLEPRPGDATDPNGTSPLHLAAKNGHIDIIRLLLQAGIDINRQTKSGTALHEAALCGKTEVVRLLLDSGINAHVRNTYSQTALDIVHQFTTSQASREIKQLLREASAALQVRATKDYCNNYDLTSLNVKAGDIITVLEQHPDGRWKGCIHDNRTGNDRVGYFPSSLGEAIVKRAGSRAGTEPSLPQGSSSSGPSAPPEEIWVLRKPFAGGDRSGSISGMAGGRGSGGHALHAGSEGVKLLATVLSQKSVSESGPGDSPAKPPEGSAGVARSQPPVAHAGQVYGEQPPKKLEPASEGKSSEAVSQWLTAFQLQLYAPNFISAGYDLPTISRMTPEDLTAIGVTKPGHRKKIAAEISGLSIPDWLPEHKPANLAVWLSMIGLAQYYKVLVDNGYENIDFITDITWEDLQEIGITKLGHQKKLMLAVRKLAELQKAEYAKYEGGPLRRKAPQSLEVMAIESPPPPEPTPADCQSPKMTTFQDSELSDELQAAMTGPAEVGPTTEKPSSHLPPTPRATTRQDSSLGGRARHMSSSQELLGDGPPGPSSPMSRSQEYLLDEGPAPGTPPREARPGRHGHSIKRASVPPVPGKPRQVLPPGTSHFTPPQTPTKTRPGSPQALGGPHGPAPATAKVKPTPQLLPPTERPMSPRSLPQSPTHRGFAYVLPQPVEGEVGPAAPGPAPPPVPTAVPTLCLPPEADAEPGRPKKRAHSLNRYAASDSEPERDELLVPAAAGPYATVQRRVGRSHSVRAPAGADKNVNRSQSFAVRPRKKGPPPPPPKRSSSALASANLADEPVPDAEPEDGLLGVRAQCRRASDLAGSVDTGSAGSVKSIAAMLELSSIGGGGRAARRPPEGHPTPRPASPEPGRVATVLASVKHKEAIGPGGEVVNRRRTLSGPVTGLLATARRGPGESADPGPFVEDGTGRQRPRGPSKGEAGVEGPPLAKVEASATLKRRIRAKQNQQENVKFILTESDTVKRRPKAKEREAGPEPPPPLSVYHNGTGTVRRRPASEQAGPPELPPPPPPAEPPPTDLAHLPPLPPPEGEARKPAKPPVSPKPVLTQPVPKLQGSPTPTSKKVPLPGPGSPEVKRAHGTPPPVSPKPPPPPTAPKPVKAVAGLPSGSAGPSPAPSPARQPPAALAKPPGTPPSLGASPAKPPSPGAPALHVPAKPPRAAAAAAAAAAAPPAPPEGASPGDSARQKLEETSACLAAALQAVEEKIRQEDAQGPRDSAAEKSTGSILDDIGSMFDDLADQLDAMLE",
# STON2_HUMAN
"MTTLDHVIATHQSEWVSFNEEPPFPAHSQGGTEEHLPGLSSSPDQSESSSGENHVVDGGSQDHSHSEQDDSSEKMGLISEAASPPGSPEQPPPDLASAISNWVQFEDDTPWASTSPPHQETAETALPLTMPCWTCPSFDSLGRCPLTSESSWTTHSEDTSSPSFGCSYTDLQLINAEEQTSGQASGADSTDNSSSLQEDEEVEMEAISWQASSPAMNGHPAPPVTSARFPSWVTFDDNEVSCPLPPVTSPLKPNTPPSASVIPDVPYNSMGSFKKRDRPKSTLMNFSKVQKLDISSLNRTPSVTEASPWRATNPFLNETLQDVQPSPINPFSAFFEEQERRSQNSSISSTTGKSQRDSLIVIYQDAISFDDSSKTQSHSDAVEKLKQLQIDDPDHFGSATLPDDDPVAWIELDAHPPGSARSQPRDGWPMMLRIPEKKNIMSSRHWGPIFVKLTDTGYLQLYYEQGLEKPFREFKLEICHEISEPRLQNYDENGRIHSLRIDRVTYKEKKKYQPKPAVAHTAEREQVIKLGTTNYDDFLSFIHAVQDRLMDLPVLSMDLSTVGLNYLEEEITVDVRDEFSGIVSKGDNQILQHHVLTRIHILSFLSGLAECRLGLNDILVKGNEIVLRQDIMPTTTTKWIKLHECRFHGCVDEDVFHNSRVILFNPLDACRFELMRFRTVFAEKTLPFTLRTATSVNGAEVEVQSWLRMSTGFSANRDPLTQVPCENVMIRYPVPSEWVKNFRRESVLGEKSLKAKVNRGASFGSTSVSGSEPVMRVTLGTAKYEHAFNSIVWRINRLPDKNSASGHPHCFFCHLELGSDREVPSRFANHVNVEFSMPTTSASKASVRSISVEDKTDVRKWVNYSAHYSYQVALGSIWLMLPTPFVHPTTLPLLFLLAMLTMFAW",
# MADD_HUMAN
"MVQKKKFCPRLLDYLVIVGARHPSSDSVAQTPELLRRYPLEDHTEFPLPPDVVFFCQPEGCLSVRQRRMSLRDDTSFVFTLTDKDTGVTRYGICVNFYRSFQKRISKEKGEGGAGSRGKEGTHATCASEEGGTESSESGSSLQPLSADSTPDVNQSPRGKRRAKAGSRSRNSTLTSLCVLSHYPFFSTFRECLYTLKRLVDCCSERLLGKKLGIPRGVQRDTMWRIFTGSLLVEEKSSALLHDLREIEAWIYRLLRSPVPVSGQKRVDIEVLPQELQPALTFALPDPSRFTLVDFPLHLPLELLGVDACLQVLTCILLEHKVVLQSRDYNALSMSVMAFVAMIYPLEYMFPVIPLLPTCMASAEQLLLAPTPYIIGVPASFFLYKLDFKMPDDVWLVDLDSNRVIAPTNAEVLPILPEPESLELKKHLKQALASMSLNTQPILNLEKFHEGQEIPLLLGRPSNDLQSTPSTEFNPLIYGNDVDSVDVATRVAMVRFFNSANVLQGFQMHTRTLRLFPRPVVAFQAGSFLASRPRQTPFAEKLARTQAVEYFGEWILNPTNYAFQRIHNNMFDPALIGDKPKWYAHQLQPIHYRVYDSNSQLAEALSVPPERDSDSEPTDDSGSDSMDYDDSSSSYSSLGDFVSEMMKCDINGDTPNVDPLTHAALGDASEVEIDELQNQKEAEEPGPDSENSQENPPLRSSSSTTASSSPSTVIHGANSEPADSTEMDDKAAVGVSKPLPSVPPSIGKSNVDRRQAEIGEGSVRRRIYDNPYFEPQYGFPPEEDEDEQGESYTPRFSQHVSGNRAQKLLRPNSLRLASDSDAESDSRASSPNSTVSNTSTEGFGGIMSFASSLYRNHSTSFSLSNLTLPTKGAREKATPFPSLKVFGLNTLMEIVTEAGPGSGEGNRRALVDQKSSVIKHSPTVKREPPSPQGRSSNSSENQQFLKEVVHSVLDGQGVGWLNMKKVRRLLESEQLRVFVLSKLNRMVQSEDDARQDIIPDVEISRKVYKGMLDLLKCTVLSLEQSYAHAGLGGMASIFGLLEIAQTHYYSKEPDKRKRSPTESVNTPVGKDPGLAGRGDPKAMAQLRVPQLGPRAPSATGKGPKELDTRSLKEENFIASIELWNKHQEVKKQKALEKQRPEVIKPVFDLGETEEKKSQISADSGVSLTSSSQRTDQDSVIGVSPAVMIRSSSQDSEVSTVVSNSSGETLGADSDLSSNAGDGPGGEGSVHLASSRGTLSDSEIETNSATSTIFGKAHSLKPSIKEKLAGSPIRTSEDVSQRVYLYEGLLGRDKGSMWDQLEDAAMETFSISKERSTLWDQMQFWEDAFLDAVMLEREGMGMDQGPQEMIDRYLSLGEHDRKRLEDDEDRLLATLLHNLISYMLLMKVNKNDIRKKVRRLMGKSHIGLVYSQQINEVLDQLANLNGRDLSIWSSGSRHMKKQTFVVHAGTDTNGDIFFMEVCDDCVVLRSNIGTVYERWWYEKLINMTYCPKTKVLCLWRRNGSETQLNKFYTKKCRELYYCVKDSMERAAARQQSIKPGPELGGEFPVQDLKTGEGGLLQVTLEGINLKFMHNQVFIELNHIKKCNTVRGVFVLEEFVPEIKEVVSHKYKTPMAHEICYSVLCLFSYVAAVHSSEEDLRTPPRPVSS",
# CNKR2_HUMAN
"MALIMEPVSKWSPSQVVDWMKGLDDCLQQYIKNFEREKISGDQLLRITHQELEDLGVSRIGHQELILEAVDLLCALNYGLETENLKTLSHKLNASAKNLQNFITGRRRSGHYDGRTSRKLPNDFLTSVVDLIGAAKSLLAWLDRSPFAAVTDYSVTRNNVIQLCLELTTIVQQDCTVYETENKILHVCKTLSGVCDHIISLSSDPLVSQSAHLEVIQLANIKPSEGLGMYIKSTYDGLHVITGTTENSPADRCKKIHAGDEVIQVNHQTVVGWQLKNLVNALREDPSGVILTLKKRPQSMLTSAPALLKNMRWKPLALQPLIPRSPTSSVATPSSTISTPTKRDSSALQDLYIPPPPAEPYIPRDEKGNLPCEDLRGHMVGKPVHKGSESPNSFLDQEYRKRFNIVEEDTVLYCYEYEKGRSSSQGRRESTPTYGKLRPISMPVEYNWVGDYEDPNKMKRDSRRENSLLRYMSNEKIAQEEYMFQRNSKKDTGKKSKKKGDKSNSPTHYSLLPSLQMDALRQDIMGTPVPETTLYHTFQQSSLQHKSKKKNKGPIAGKSKRRISCKDLGRGDCEGWLWKKKDAKSYFSQKWKKYWFVLKDASLYWYINEEDEKAEGFISLPEFKIDRASECRKKYAFKACHPKIKSFYFAAEHLDDMNRWLNRINMLTAGYAERERIKQEQDYWSESDKEEADTPSTPKQDSPPPPYDTYPRPPSMSCASPYVEAKHSRLSSTETSQSQSSHEEFRQEVTGSSAVSPIRKTASQRRSWQDLIETPLTSSGLHYLQTLPLEDSVFSDSAAISPEHRRQSTLPTQKCHLQDHYGPYPLAESERMQVLNGNGGKPRSFTLPRDSGFNHCCLNAPVSACDPQDDVQPPEVEEEEEEEEEEGEAAGENIGEKSESREEKLGDSLQDLYRALEQASLSPLGEHRISTKMEYKLSFIKRCNDPVMNEKLHRLRILKSTLKAREGEVAIIDKVLDNPDLTSKEFQQWKQMYLDLFLDICQNTTSNDPLSISSEVDVITSSLAHTHSYIETHV",
# CCG8_HUMAN
"MESLKRWNEERGLWCEKGVQVLLTTVGAFAAFGLMTIAISTDYWLYTRALICNTTNLTAGGDDGTPHRGGGGASEKKDPGGLTHSGLWRICCLEGLKRGVCVKINHFPEDTDYDHDSAEYLLRVVRASSIFPILSAILLLLGGVCVAASRVYKSKRNIILGAGILFVAAGLSNIIGVIVYISANAGEPGPKRDEEKKNHYSYGWSFYFGGLSFILAEVIGVLAVNIYIERSREAHCQSRSDLLKAGGGAGGSGGSGPSAILRLPSYRFRYRRRSRSSSRSSEPSPSRDASPGGPGGPGFASTDISMYTLSRDPSKGSVAAGLAGAGGGGGGAVGAFGGAAGGAGGGGGGGGGAGAERDRGGASGFLTLHNAFPKEAGGGVTVTVTGPPAPPAPAPPAPSAPAPGTLAKEAAASNTNTLNRKTTPV",
# NANO1_HUMAN
"MEAFPWAPRSPRRGRAPPPMALVPSARYVSAPGPAHPQPFSSWNDYLGLATLITKAVDGEPRFGCARGGNGGGGSPPSSSSSSCCSPHTGAGPGALGPALGPPDYDEDDDDDSDEPGSRGRYLGSALELRALELCAGPAEAGLLEERFAELSPFAGRAAAVLLGCAPAAAAAATTTSEATPREERAPAWAAEPRLHAASGAAAARLLKPELQVCVFCRNNKEAMALYTTHILKGPDGRVLCPVLRRYTCPLCGASGDNAHTIKYCPLSKVPPPPARPPPRSARDGPPGKKLR",
# DGCR8_HUMAN
"METDESPSPLPCGPAGEAVMESRARPFQALPREQSPPPPLQTSSGAEVMDVGSGGDGQSELPAEDPFNFYGASLLSKGSFSKGRLLIDPNCSGHSPRTARHAPAVRKFSPDLKLLKDVKISVSFTESCRSKDRKVLYTGAERDVRAECGLLLSPVSGDVHACPFGGSVGDGVGIGGESADKKDEENELDQEKRVEYAVLDELEDFTDNLELDEEGAGGFTAKAIVQRDRVDEEALNFPYEDDFDNDVDALLEEGLCAPKKRRTEEKYGGDSDHPSDGETSVQPMMTKIKTVLKSRGRPPTEPLPDGWIMTFHNSGVPVYLHRESRVVTWSRPYFLGTGSIRKHDPPLSSIPCLHYKKMKDNEEREQSSDLTPSGDVSPVKPLSRSAELEFPLDEPDSMGADPGPPDEKDPLGAEAAPGALGQVKAKVEVCKDESVDLEEFRSYLEKRFDFEQVTVKKFRTWAERRQFNREMKRKQAESERPILPANQKLITLSVQDAPTKKEFVINPNGKSEVCILHEYMQRVLKVRPVYNFFECENPSEPFGASVTIDGVTYGSGTASSKKLAKNKAARATLEILIPDFVKQTSEEKPKDSEELEYFNHISIEDSRVYELTSKAGLLSPYQILHECLKRNHGMGDTSIKFEVVPGKNQKSEYVMACGKHTVRGWCKNKRVGKQLASQKILQLLHPHVKNWGSLLRMYGRESSKMVKQETSDKSVIELQQYAKKNKPNLHILSKLQEEMKRLAEEREETRKKPKMSIVASAQPGGEPLCTVDV",
# CTTB2_HUMAN
"MATDGASCEPDLSRAPEDAAGAAAEAAKKEFDVDTLSKSELRMLLSVMEGELEARDLVIEALRARRKEVFIQERYGRFNLNDPFLALQRDYEAGAGDKEKKPVCTNPLSILEAVMAHCKKMQERMSAQLAAAESRQKKLEMEKLQLQALEQEHKKLAARLEEERGKNKQVVLMLVKECKQLSGKVIEEAQKLEDVMAKLEEEKKKTNELEEELSAEKRRSTEMEAQMEKQLSEFDTEREQLRAKLNREEAHTTDLKEEIDKMRKMIEQLKRGSDSKPSLSLPRKTKDRRLVSISVGTEGTVTRSVACQTDLVTENADHMKKLPLTMPVKPSTGSPLVSANAKGSVCTSATMARPGIDRQASYGDLIGASVPAFPPPSANKIEENGPSTGSTPDPTSSTPPLPSNAAPPTAQTPGIAPQNSQAPPMHSLHSPCANTSLHPGLNPRIQAARFRFQGNANDPDQNGNTTQSPPSRDVSPTSRDNLVAKQLARNTVTQALSRFTSPQAGAPSRPGVPPTGDVGTHPPVGRTSLKTHGVARVDRGNPPPIPPKKPGLSQTPSPPHPQLKVIIDSSRASNTGAKVDNKTVASTPSSLPQGNRVINEENLPKSSSPQLPPKPSIDLTVAPAGCAVSALATSQVGAWPAATPGLNQPACSDSSLVIPTTIAFCSSINPVSASSCRPGASDSLLVTASGWSPSLTPLLMSGGPAPLAGRPTLLQQAAAQGNVTLLSMLLNEEGLDINYSCEDGHSALYSAAKNGHTDCVRLLLSAEAQVNAADKNGFTPLCAAAAQGHFECVELLISYDANINHAADGGQTPLYLACKNGNKECIKLLLEAGTNRSVKTTDGWTPVHAAVDTGNVDSLKLLMYHRIPAHGNSFNEEESESSVFDLDGGEESPEGISKPVVPADLINHANREGWTAAHIAASKGFKNCLEILCRHGGLEPERRDKCNRTVHDVATDDCKHLLENLNALKIPLRISVGEIEPSNYGSDDLECENTICALNIRKQTSWDDFSKAVSQALTNHFQAISSDGWWSLEDVTCNNTTDSNIGLSARSIRSITLGNVPWSVGQSFAQSPWDFMRKNKAEHITVLLSGPQEGCLSSVTYASMIPLQMMQNYLRLVEQYHNVIFHGPEGSLQDYIVHQLALCLKHRQMAAGFSCEIVRAEVDAGFSKEQLLDLFISSACLIPVKQSPSKKKIIIILENLEKSSLSELLRDFLAPLENRSTESPCTFQKGNGLSECYYFHENCFLMGTIAKACLQGSDLLVQQHFRWVQLRWDGEPMQGLLQRFLRRKVVNKFKGQAPSPCDPVCKIVDWALSVWRQLNSCLARLGTPEALLGPKYFLSCPVVPGHAQVTVKWMSKLWNGVIAPRVQEAILSRASVKRQPGFGQTTAKRHPSQGQQAVVKAALSILLNKAVLHGCPLPRAELDQHTADFKGGSFPLSIVSSYNTCNKKKGESGAWRKVNTSPRRKSGRFSLPTWNKPDLSTEGMKNKTISQLNCNRNASLSKQKSLENDLSLTLNLDQRLSLGSDDEADLVKELQSMCSSKSESDISKIADSRDDLRMFDSSGNNPVLSATINNLRMPVSQKEVSPLSSHQTTECSNSKSKTELGVSRVKSFLPVPRSKVTQCSQNTKRSSSSSNTRQIEINNNSKEVNWNLHKNEHLEKPNK",
# RPGF4_HUMAN
"MVAAHAAHSSSSAEWIACLDKRPLERSSEDVDIIFTRLKEVKAFEKFHPNLLHQICLCGYYENLEKGITLFRQGDIGTNWYAVLAGSLDVKVSETSSHQDAVTICTLGIGTAFGESILDNTPRHATIVTRESSELLRIEQKDFKALWEKYRQYMAGLLAPPYGVMETGSNNDRIPDKENTPLIEPHVPLRPANTITKVPSEKILRAGKILRNAILSRAPHMIRDRKYHLKTYRQCCVGTELVDWMMQQTPCVHSRTQAVGMWQVLLEDGVLNHVDQEHHFQDKYLFYRFLDDEHEDAPLPTEEEKKECDEELQDTMLLLSQMGPDAHMRMILRKPPGQRTVDDLEIIYEELLHIKALSHLSTTVKRELAGVLIFESHAKGGTVLFNQGEEGTSWYIILKGSVNVVIYGKGVVCTLHEGDDFGKLALVNDAPRAASIVLREDNCHFLRVDKEDFNRILRDVEANTVRLKEHDQDVLVLEKVPAGNRASNQGNSQPQQKYTVMSGTPEKILEHFLETIRLEATLNEATDSVLNDFIMMHCVFMPNTQLCPALVAHYHAQPSQGTEQEKMDYALNNKRRVIRLVLQWAAMYGDLLQEDDVSMAFLEEFYVSVSDDARMIAALKEQLPELEKIVKQISEDAKAPQKKHKVLLQQFNTGDERAQKRQPIRGSDEVLFKVYCMDHTYTTIRVPVATSVKEVISAVADKLGSGEGLIIVKMSSGGEKVVLKPNDVSVFTTLTINGRLFACPREQFDSLTPLPEQEGPTVGTVGTFELMSSKDLAYQMTIYDWELFNCVHELELIYHTFGRHNFKKTTANLDLFLRRFNEIQFWVVTEICLCSQLSKRVQLLKKFIKIAAHCKEYKNLNSFFAIVMGLSNVAVSRLALTWEKLPSKFKKFYAEFESLMDPSRNHRAYRLTVAKLEPPLIPFMPLLIKDMTFTHEGNKTFIDNLVNFEKMRMIANTARTVRYYRSQPFNPDAAQANKNHQDVRSYVRQLNVIDNQRTLSQMSHRLEPRRP",
# NICA_HUMAN
"MATAGGGSGADPGSRGLLRLLSFCVLLAGLCRGNSVERKIYIPLNKTAPCVRLLNATHQIGCQSSISGDTGVIHVVEKEEDLQWVLTDGPNPPYMVLLESKHFTRDLMEKLKGRTSRIAGLAVSLTKPSPASGFSPSVQCPNDGFGVYSNSYGPEFAHCREIQWNSLGNGLAYEDFSFPIFLLEDENETKVIKQCYQDHNLSQNGSAPTFPLCAMQLFSHMHAVISTATCMRRSSIQSTFSINPEIVCDPLSDYNVWSMLKPINTTGTLKPDDRVVVAATRLDSRSFFWNVAPGAESAVASFVTQLAAAEALQKAPDVTTLPRNVMFVFFQGETFDYIGSSRMVYDMEKGKFPVQLENVDSFVELGQVALRTSLELWMHTDPVSQKNESVRNQVEDLLATLEKSGAGVPAVILRRPNQSQPLPPSSLQRFLRARNISGVVLADHSGAFHNKYYQSIYDTAENINVSYPEWLSPEEDLNFVTDTAKALADVATVLGRALYELAGGTNFSDTVQADPQTVTRLLYGFLIKANNSWFQSILRQDLRSYLGDGPLQHYIAVSSPTNTTYVVQYALANLTGTVVNLTREQCQDPSKVPSENKDLYEYSWVQGPLHSNETDRLPRCVRSTARLARALSPAFELSQWSSTEYSTWTESRWKDIRARIFLIASKELELITLTVGFGILIFSLIVTYCINAKADVLFIAPREPGAVSY",
# ELMO1_HUMAN
"MPPPADIVKVAIEWPGAYPKLMEIDQKKPLSAIIKEVCDGWSLANHEYFALQHADSSNFYITEKNRNEIKNGTILRLTTSPAQNAQQLHERIQSSSMDAKLEALKDLASLSRDVTFAQEFINLDGISLLTQMVESGTERYQKLQKIMKPCFGDMLSFTLTAFVELMDHGIVSWDTFSVAFIKKIASFVNKSAIDISILQRSLAILESMVLNSHDLYQKVAQEITIGQLIPHLQGSDQEIQTYTIAVINALFLKAPDERRQEMANILAQKQLRSIILTHVIRAQRAINNEMAHQLYVLQVLTFNLLEDRMMTKMDPQDQAQRDIIFELRRIAFDAESEPNNSSGSMEKRKSMYTRDYKKLGFINHVNPAMDFTQTPPGMLALDNMLYFAKHHQDAYIRIVLENSSREDKHECPFGRSSIELTKMLCEILKVGELPSETCNDFHPMFFTHDRSFEEFFCICIQLLNKTWKEMRATSEDFNKVMQVVKEQVMRALTTKPSSLDQFKSKLQNLSYTEILKIRQSERMNQEDFQSRPILELKEKIQPEILELIKQQRLNRLVEGTCFRKLNARRRQDKFWYCRLSPNHKVLHYGDLEESPQGEVPHDSLQDKLPVADIKAVVTGKDCPHMKEKGALKQNKEVLELAFSILYDSNCQLNFIAPDKHEYCIWTDGLNALLGKDMMSDLTRNDLDTLLSMEIKLRLLDLENIQIPDAPPPIPKEPSNYDFVYDCN",
# WASF1_HUMAN
"MPLVKRNIDPRHLCHTALPRGIKNELECVTNISLANIIRQLSSLSKYAEDIFGELFNEAHSFSFRVNSLQERVDRLSVSVTQLDPKEEELSLQDITMRKAFRSSTIQDQQLFDRKTLPIPLQETYDVCEQPPPLNILTPYRDDGKEGLKFYTNPSYFFDLWKEKMLQDTEDKRKEKRKQKQKNLDRPHEPEKVPRAPHDRRREWQKLAQGPELAEDDANLLHKHIEVANGPASHFETRPQTYVDHMDGSYSLSALPFSQMSELLTRAEERVLVRPHEPPPPPPMHGAGDAKPIPTCISSATGLIENRPQSPATGRTPVFVSPTPPPPPPPLPSALSTSSLRASMTSTPPPPVPPPPPPPATALQAPAVPPPPAPLQIAPGVLHPAPPPIAPPLVQPSPPVARAAPVCETVPVHPLPQGEVQGLPPPPPPPPLPPPGIRPSSPVTVTALAHPPSGLHPTPSTAPGPHVPLMPPSPPSQVIPASEPKRHPSTLPVISDARSVLLEAIRKGIQLRKVEEQREQEAKHERIENDVATILSRRIAVEYSDSEDDSEFDEVDWLE",
# AP3S1_HUMAN
"MIKAILIFNNHGKPRLSKFYQPYSEDTQQQIIRETFHLVSKRDENVCNFLEGGLLIGGSDNKLIYRHYATLYFVFCVDSSESELGILDLIQVFVETLDKCFENVCELDLIFHVDKVHNILAEMVMGGMVLETNMNEIVTQIDAQNKLEKSEAGLAGAPARAVSAVKNMNLPEIPRNINIGDISIKVPNLPSFK",
# TSC1_HUMAN
"MAQQANVGELLAMLDSPMLGVRDDVTAVFKENLNSDRGPMLVNTLVDYYLETSSQPALHILTTLQEPHDKHLLDRINEYVGKAATRLSILSLLGHVIRLQPSWKHKLSQAPLLPSLLKCLKMDTDVVVLTTGVLVLITMLPMIPQSGKQHLLDFFDIFGRLSSWCLKKPGHVAEVYLVHLHASVYALFHRLYGMYPCNFVSFLRSHYSMKENLETFEEVVKPMMEHVRIHPELVTGSKDHELDPRRWKRLETHDVVIECAKISLDPTEASYEDGYSVSHQISARFPHRSADVTTSPYADTQNSYGCATSTPYSTSRLMLLNMPGQLPQTLSSPSTRLITEPPQATLWSPSMVCGMTTPPTSPGNVPPDLSHPYSKVFGTTAGGKGTPLGTPATSPPPAPLCHSDDYVHISLPQATVTPPRKEERMDSARPCLHRQHHLLNDRGSEEPPGSKGSVTLSDLPGFLGDLASEEDSIEKDKEEAAISRELSEITTAEAEPVVPRGGFDSPFYRDSLPGSQRKTHSAASSSQGASVNPEPLHSSLDKLGPDTPKQAFTPIDLPCGSADESPAGDRECQTSLETSIFTPSPCKIPPPTRVGFGSGQPPPYDHLFEVALPKTAHHFVIRKTEELLKKAKGNTEEDGVPSTSPMEVLDRLIQQGADAHSKELNKLPLPSKSVDWTHFGGSPPSDEIRTLRDQLLLLHNQLLYERFKRQQHALRNRRLLRKVIKAAALEEHNAAMKDQLKLQEKDIQMWKVSLQKEQARYNQLQEQRDTMVTKLHSQIRQLQHDREEFYNQSQELQTKLEDCRNMIAELRIELKKANNKVCHTELLLSQVSQKLSNSESVQQQMEFLNRQLLVLGEVNELYLEQLQNKHSDTTKEVEMMKAAYRKELEKNRSHVLQQTQRLDTSQKRILELESHLAKKDHLLLEQKKYLEDVKLQARGQLQAAESRYEAQKRITQVFELEILDLYGRLEKDGLLKKLEEEKAEAAEAAEERLDCCNDGCSDSMVGHNEEASGHNGETKTPRPSSARGSSGSRGGGGSSSSSSELSTPEKPPHQRAGPFSSRWETTMGEASASIPTTVGSLPSSKSFLGMKARELFRNKSESQCDEDGMTSSLSESLKTELGKDLGVEAKIPLNLDGPHPSPPTPDSVGQLHIMDYNETHHEHS",
# SL9A6_HUMAN
"MARRGWRRAPLRRGVGSSPRARRLMRPLWLLLAVGVFDWAGASDGGGGEARAMDEEIVSEKQAEESHRQDSANLLIFILLLTLTILTIWLFKHRRARFLHETGLAMIYGLLVGLVLRYGIHVPSDVNNVTLSCEVQSSPTTLLVNVSGKFYEYMLKGEISSHELNNVQDNEMLRKVTFDPEVFFNILLPPIIFYAGYSLKRRHFFRNLGSILAYAFLGTAISCFVIGSIMYGCVTLMKVTGQLAGDFYFTDCLLFGAIVSATDPVTVLAIFHELQVDVELYALLFGESVLNDAVAIVLSSSIVAYQPAGDNSHTFDVTAMFKSIGIFLGIFSGSFAMGAATGVVTALVTKFTKLREFQLLETGLFFLMSWSTFLLAEAWGFTGVVAVLFCGITQAHYTYNNLSTESQHRTKQLFELLNFLAENFIFSYMGLTLFTFQNHVFNPTFVVGAFVAIFLGRAANIYPLSLLLNLGRRSKIGSNFQHMMMFAGLRGAMAFALAIRDTATYARQMMFSTTLLIVFFTVWVFGGGTTAMLSCLHIRVGVDSDQEHLGVPENERRTTKAESAWLFRMWYNFDHNYLKPLLTHSGPPLTTTLPACCGPIARCLTSPQAYENQEQLKDDDSDLILNDGDISLTYGDSTVNTEPATSSAPRRFMGNSSEDALDRELAFGDHELVIRGTRLVLPMDDSEPPLNLLDNTRHGPA",
# SEPT8_HUMAN
"MAATDLERFSNAEPEPRSLSLGGHVGFDSLPDQLVSKSVTQGFSFNILCVGETGIGKSTLMNTLFNTTFETEEASHHEACVRLRPQTYDLQESNVQLKLTIVDAVGFGDQINKDESYRPIVDYIDAQFENYLQEELKIRRSLFDYHDTRIHVCLYFITPTGHSLKSLDLVTMKKLDSKVNIIPIIAKADTISKSELHKFKIKIMGELVSNGVQIYQFPTDDEAVAEINAVMNAHLPFAVVGSTEEVKVGNKLVRARQYPWGVVQVENENHCDFVKLREMLIRVNMEDLREQTHSRHYELYRRCKLEEMGFQDSDGDSQPFSLQETYEAKRKEFLSELQRKEEEMRQMFVNKVKETELELKEKERELHEKFEHLKRVHQEEKRKVEEKRRELEEETNAFNRRKAAVEALQSQALHATSQQPLRKDKDKKNRSDIGAHQPGMSLSSSKVMMTKASVEPLNCSSWWPAIQCCSCLVRDATWREGFL",
# LPAR1_HUMAN
"MAAISTSIPVISQPQFTAMNEPQCFYNESIAFFYNRSGKHLATEWNTVSKLVMGLGITVCIFIMLANLLVMVAIYVNRRFHFPIYYLMANLAAADFFAGLAYFYLMFNTGPNTRRLTVSTWLLRQGLIDTSLTASVANLLAIAIERHITVFRMQLHTRMSNRRVVVVIVVIWTMAIVMGAIPSVGWNCICDIENCSNMAPLYSDSYLVFWAIFNLVTFVVMVVLYAHIFGYVRQRTMRMSRHSSGPRRNRDTMMSLLKTVVIVLGAFIICWTPGLVLLLLDVCCPQCDVLAYEKFFLLLAEFNSAMNPIIYSYRDKEMSATFRQILCCQRSENPTGPTEGSDRSASSLNHTILAGVHSNDHSVV",
# AKAP1_HUMAN
"MAIQFRSLFPLALPGMLALLGWWWFFSRKKGHVSSHDEQQVEAGAVQLRADPAIKEPLPVEDVCPKVVSTPPSVTEPPEKELSTVSKLPAEPPALLQTHPPCRRSESSGILPNTTDMRLRPGTRRDDSTKLELALTGGEAKSIPLECPLSSPKGVLFSSKSAEVCKQDSPFSRVPRKVQPGYPVVPAEKRSSGERARETGGAEGTGDAVLGEKVLEEALLSREHVLELENSKGPSLASLEGEEDKGKSSSSQVVGPVQEEEYVAEKLPSRFIESAHTELAKDDAAPAPPVADAKAQDRGVEGELGNEESLDRNEEGLDRNEEGLDRNEESLDRNEEGLDRNEEIKRAAFQIISQVISEATEQVLATTVGKVAGRVCQASQLQGQKEESCVPVHQKTVLGPDTAEPATAEAAVAPPDAGLPLPGLPAEGSPPPKTYVSCLKSLLSSPTKDSKPNISAHHISLASCLALTTPSEELPDRAGILVEDATCVTCMSDSSQSVPLVASPGHCSDSFSTSGLEDSCTETSSSPRDKAITPPLPESTVPFSNGVLKGELSDLGAEDGWTMDAEADHSGGSDRNSMDSVDSCCSLKKTESFQNAQAGSNPKKVDLIIWEIEVPKHLVGRLIGKQGRYVSFLKQTSGAKIYISTLPYTQSVQICHIEGSQHHVDKALNLIGKKFKELNLTNIYAPPLPSLALPSLPMTSWLMLPDGITVEVIVVNQVNAGHLFVQQHTHPTFHALRSLDQQMYLCYSQPGIPTLPTPVEITVICAAPGADGAWWRAQVVASYEETNEVEIRYVDYGGYKRVKVDVLRQIRSDFVTLPFQGAEVLLDSVMPLSDDDQFSPEADAAMSEMTGNTALLAQVTSYSPTGLPLIQLWSVVGDEVVLINRSLVERGLAQWVDSYYTSL",
# NEUG_HUMAN
"MDCCTENACSKPDDDILDIPLDDPGANAAAAKIQASFRGHMARKKIKSGERGRKGPGPGGPGGAGVARGGAGGGPSGD",
# PTPRU_HUMAN
"MARAQALVLALTFQLCAPETETPAAGCTFEEASDPAVPCEYSQAQYDDFQWEQVRIHPGTRAPADLPHGSYLMVNTSQHAPGQRAHVIFQSLSENDTHCVQFSYFLYSRDGHSPGTLGVYVRVNGGPLGSAVWNMTGSHGRQWHQAELAVSTFWPNEYQVLFEALISPDRRGYMGLDDILLLSYPCAKAPHFSRLGDVEVNAGQNASFQCMAAGRAAEAERFLLQRQSGALVPAAGVRHISHRRFLATFPLAAVSRAEQDLYRCVSQAPRGAGVSNFAELIVKEPPTPIAPPQLLRAGPTYLIIQLNTNSIIGDGPIVRKEIEYRMARGPWAEVHAVSLQTYKLWHLDPDTEYEISVLLTRPGDGGTGRPGPPLISRTKCAEPMRAPKGLAFAEIQARQLTLQWEPLGYNVTRCHTYTVSLCYHYTLGSSHNQTIRECVKTEQGVSRYTIKNLLPYRNVHVRLVLTNPEGRKEGKEVTFQTDEDVPSGIAAESLTFTPLEDMIFLKWEEPQEPNGLITQYEISYQSIESSDPAVNVPGPRRTISKLRNETYHVFSNLHPGTTYLFSVRARTGKGFGQAALTEITTNISAPSFDYADMPSPLGESENTITVLLRPAQGRGAPISVYQVIVEEERARRLRREPGGQDCFPVPLTFEAALARGLVHYFGAELAASSLPEAMPFTVGDNQTYRGFWNPPLEPRKAYLIYFQAASHLKGETRLNCIRIARKAACKESKRPLEVSQRSEEMGLILGICAGGLAVLILLLGAIIVIIRKGRDHYAYSYYPKPVNMTKATVNYRQEKTHMMSAVDRSFTDQSTLQEDERLGLSFMDTHGYSTRGDQRSGGVTEASSLLGGSPRRPCGRKGSPYHTGQLHPAVRVADLLQHINQMKTAEGYGFKQEYESFFEGWDATKKKDKVKGSRQEPMPAYDRHRVKLHPMLGDPNADYINANYIDGYHRSNHFIATQGPKPEMVYDFWRMVWQEHCSSIVMITKLVEVGRVKCSRYWPEDSDTYGDIKIMLVKTETLAEYVVRTFALERRGYSARHEVRQFHFTAWPEHGVPYHATGLLAFIRRVKASTPPDAGPIVIHCSAGTGRTGCYIVLDVMLDMAECEGVVDIYNCVKTLCSRRVNMIQTEEQYIFIHDAILEACLCGETTIPVSEFKATYKEMIRIDPQSNSSQLREEFQTLNSVTPPLDVEECSIALLPRNRDKNRSMDVLPPDRCLPFLISTDGDSNNYINAALTDSYTRSAAFIVTLHPLQSTTPDFWRLVYDYGCTSIVMLNQLNQSNSAWPCLQYWPEPGRQQYGLMEVEFMSGTADEDLVARVFRVQNISRLQEGHLLVRHFQFLRWSAYRDTPDSKKAFLHLLAEVDKWQAESGDGRTIVHCLNGGGRSGTFCACATVLEMIRCHNLVDVFFAAKTLRNYKPNMVETMDQYHFCYDVALEYLEGLESR",
# ARC1A_HUMAN
"MSLHQFLLEPITCHAWNRDRTQIALSPNNHEVHIYKKNGSQWVKAHELKEHNGHITGIDWAPKSDRIVTCGADRNAYVWSQKDGVWKPTLVILRINRAATFVKWSPLENKFAVGSGARLISVCYFESENDWWVSKHIKKPIRSTVLSLDWHPNNVLLAAGSCDFKCRVFSAYIKEVDEKPASTPWGSKMPFGQLMSEFGGSGTGGWVHGVSFSASGSRLAWVSHDSTVSVADASKSVQVSTLKTEFLPLLSVSFVSENSVVAAGHDCCPMLFNYDDRGCLTFVSKLDIPKQSIQRNMSAMERFRNMDKRATTEDRNTALETLHQNSITQVSIYEVDKQDCRKFCTTGIDGAMTIWDFKTLESSIQGLRIM",
# TENR_HUMAN
"MGADGETVVLKNMLIGINLILLGSMIKPSECQLEVTTERVQRQSVEEEGGIANYNTSSKEQPVVFNHVYNINVPLDNLCSSGLEASAEQEVSAEDETLAEYMGQTSDHESQVTFTHRINFPKKACPCASSAQVLQELLSRIEMLEREVSVLRDQCNANCCQESAATGQLDYIPHCSGHGNFSFESCGCICNEGWFGKNCSEPYCPLGCSSRGVCVDGQCICDSEYSGDDCSELRCPTDCSSRGLCVDGECVCEEPYTGEDCRELRCPGDCSGKGRCANGTCLCEEGYVGEDCGQRQCLNACSGRGQCEEGLCVCEEGYQGPDCSAVAPPEDLRVAGISDRSIELEWDGPMAVTEYVISYQPTALGGLQLQQRVPGDWSGVTITELEPGLTYNISVYAVISNILSLPITAKVATHLSTPQGLQFKTITETTVEVQWEPFSFSFDGWEISFIPKNNEGGVIAQVPSDVTSFNQTGLKPGEEYIVNVVALKEQARSPPTSASVSTVIDGPTQILVRDVSDTVAFVEWIPPRAKVDFILLKYGLVGGEGGRTTFRLQPPLSQYSVQALRPGSRYEVSVSAVRGTNESDSATTQFTTEIDAPKNLRVGSRTATSLDLEWDNSEAEVQEYKVVYSTLAGEQYHEVLVPRGIGPTTRATLTDLVPGTEYGVGISAVMNSQQSVPATMNARTELDSPRDLMVTASSETSISLIWTKASGPIDHYRITFTPSSGIASEVTVPKDRTSYTLTDLEPGAEYIISVTAERGRQQSLESTVDAFTGFRPISHLHFSHVTSSSVNITWSDPSPPADRLILNYSPRDEEEEMMEVSLDATKRHAVLMGLQPATEYIVNLVAVHGTVTSEPIVGSITTGIDPPKDITISNVTKDSVMVSWSPPVASFDYYRVSYRPTQVGRLDSSVVPNTVTEFTITRLNPATEYEISLNSVRGREESERICTLVHTAMDNPVDLIATNITPTEALLQWKAPVGEVENYVIVLTHFAVAGETILVDGVSEEFRLVDLLPSTHYTATMYATNGPLTSGTISTNFSTLLDPPANLTASEVTRQSALISWQPPRAEIENYVLTYKSTDGSRKELIVDAEDTWIRLEGLLENTDYTVLLQAAQDTTWSSITSTAFTTGGRVFPHPQDCAQHLMNGDTLSGVYPIFLNGELSQKLQVYCDMTTDGGGWIVFQRRQNGQTDFFRKWADYRVGFGNVEDEFWLGLDNIHRITSQGRYELRVDMRDGQEAAFASYDRFSVEDSRNLYKLRIGSYNGTAGDSLSYHQGRPFSTEDRDNDVAVTNCAMSYKGAWWYKNCHRTNLNGKYGESRHSQGINWYHWKGHEFSIPFVEMKMRPYNHRLMAGRKRQSLQF",
# SYN2_HUMAN
"MMNFLRRRLSDSSFIANLPNGYMTDLQRPEPQQPPPPPPPGPGAASASAAPPTASPGPERRPPPASAPAPQPAPTPSVGSSFFSSLSQAVKQTAASAGLVDAPAPAPAAARKAKVLLVVDEPHADWAKCFRGKKVLGDYDIKVEQAEFSELNLVAHADGTYAVDMQVLRNGTKVVRSFRPDFVLIRQHAFGMAENEDFRHLIIGMQYAGLPSINSLESIYNFCDKPWVFAQLVAIYKTLGGEKFPLIEQTYYPNHKEMLTLPTFPVVVKIGHAHSGMGKVKVENHYDFQDIASVVALTQTYATAEPFIDSKYDIRVQKIGNNYKAYMRTSISGNWKTNTGSAMLEQIAMSDRYKLWVDTCSEMFGGLDICAVKAVHGKDGKDYIFEVMDCSMPLIGEHQVEDRQLITELVISKMNQLLSRTPALSPQRPLTTQQPQSGTLKDPDSSKTPPQRPPPQGGPGQPQGMQPPGKVLPPRRLPPGPSLPPSSSSSSSSSSSAPQRPGGPTTHGDAPSSSSSLAEAQPPLAAPPQKPQPHPQLNKSQSLTNAFSFSESSFFRSSANEDEAKAETIRSLRKSFASLFSD",
# DLG3_HUMAN
"MHKHQHCCKCPECYEVTRLAALRRLEPPGYGDWQVPDPYGPGGGNGASAGYGGYSSQTLPSQAGATPTPRTKAKLIPTGRDVGPVPPKPVPGKSTPKLNGSGPSWWPECTCTNRDWYEQVNGSDGMFKYEEIVLERGNSGLGFSIAGGIDNPHVPDDPGIFITKIIPGGAAAMDGRLGVNDCVLRVNEVDVSEVVHSRAVEALKEAGPVVRLVVRRRQPPPETIMEVNLLKGPKGLGFSIAGGIGNQHIPGDNSIYITKIIEGGAAQKDGRLQIGDRLLAVNNTNLQDVRHEEAVASLKNTSDMVYLKVAKPGSLHLNDMYAPPDYASTFTALADNHISHNSSLGYLGAVESKVSYPAPPQVPPTRYSPIPRHMLAEEDFTREPRKIILHKGSTGLGFNIVGGEDGEGIFVSFILAGGPADLSGELRRGDRILSVNGVNLRNATHEQAAAALKRAGQSVTIVAQYRPEEYSRFESKIHDLREQMMNSSMSSGSGSLRTSEKRSLYVRALFDYDRTRDSCLPSQGLSFSYGDILHVINASDDEWWQARLVTPHGESEQIGVIPSKKRVEKKERARLKTVKFHARTGMIESNRDFPGLSDDYYGAKNLKGQEDAILSYEPVTRQEIHYARPVIILGPMKDRVNDDLISEFPHKFGSCVPHTTRPRRDNEVDGQDYHFVVSREQMEKDIQDNKFIEAGQFNDNLYGTSIQSVRAVAERGKHCILDVSGNAIKRLQQAQLYPIAIFIKPKSIEALMEMNRRQTYEQANKIYDKAMKLEQEFGEYFTAIVQGDSLEEIYNKIKQIIEDQSGHYIWVPSPEKL",
# KCNJ9_HUMAN
"MAQENAAFSPGQEEPPRRRGRQRYVEKDGRCNVQQGNVRETYRYLTDLFTTLVDLQWRLSLLFFVLAYALTWLFFGAIWWLIAYGRGDLEHLEDTAWTPCVNNLNGFVAAFLFSIETETTIGYGHRVITDQCPEGIVLLLLQAILGSMVNAFMVGCMFVKISQPNKRAATLVFSSHAVVSLRDGRLCLMFRVGDLRSSHIVEASIRAKLIRSRQTLEGEFIPLHQTDLSVGFDTGDDRLFLVSPLVISHEIDAASPFWEASRRALERDDFEIVVILEGMVEATGMTCQARSSYLVDEVLWGHRFTSVLTLEDGFYEVDYASFHETFEVPTPSCSARELAEAAARLDAHLYWSIPSRLDEKVEEEGAGEGAGGEAGADKEQNGCLPPPESESKV",
# NRCAM_HUMAN
"MQLKIMPKKKRLSAGRVPLILFLCQMISALEVPLDPKLLEDLVQPPTITQQSPKDYIIDPRENIVIQCEAKGKPPPSFSWTRNGTHFDIDKDPLVTMKPGTGTLIINIMSEGKAETYEGVYQCTARNERGAAVSNNIVVRPSRSPLWTKEKLEPITLQSGQSLVLPCRPPIGLPPPIIFWMDNSFQRLPQSERVSQGLNGDLYFSNVLPEDTREDYICYARFNHTQTIQQKQPISVKVISVDELNDTIAANLSDTEFYGAKSSRERPPTFLTPEGNASNKEELRGNVLSLECIAEGLPTPIIYWAKEDGMLPKNRTVYKNFEKTLQIIHVSEADSGNYQCIAKNALGAIHHTISVRVKAAPYWITAPQNLVLSPGEDGTLICRANGNPKPRISWLTNGVPIEIAPDDPSRKIDGDTIIFSNVQERSSAVYQCNASNEYGYLLANAFVNVLAEPPRILTPANTLYQVIANRPALLDCAFFGSPLPTIEWFKGAKGSALHEDIYVLHENGTLEIPVAQKDSTGTYTCVARNKLGMAKNEVHLEIKDPTWIVKQPEYAVVQRGSMVSFECKVKHDHTLSLTVLWLKDNRELPSDERFTVDKDHLVVADVSDDDSGTYTCVANTTLDSVSASAVLSVVAPTPTPAPVYDVPNPPFDLELTDQLDKSVQLSWTPGDDNNSPITKFIIEYEDAMHKPGLWHHQTEVSGTQTTAQLKLSPYVNYSFRVMAVNSIGKSLPSEASEQYLTKASEPDKNPTAVEGLGSEPDNLVITWKPLNGFESNGPGLQYKVSWRQKDGDDEWTSVVVANVSKYIVSGTPTFVPYLIKVQALNDMGFAPEPAVVMGHSGEDLPMVAPGNVRVNVVNSTLAEVHWDPVPLKSIRGHLQGYRIYYWKTQSSSKRNRRHIEKKILTFQGSKTHGMLPGLEPFSHYTLNVRVVNGKGEGPASPDRVFNTPEGVPSAPSSLKIVNPTLDSLTLEWDPPSHPNGILTEYTLKYQPINSTHELGPLVDLKIPANKTRWTLKNLNFSTRYKFYFYAQTSAGSGSQITEEAVTTVDEAGILPPDVGAGKVQAVNPRISNLTAAAAETYANISWEYEGPEHVNFYVEYGVAGSKEEWRKEIVNGSRSFFGLKGLMPGTAYKVRVGAVGDSGFVSSEDVFETGPAMASRQVDIATQGWFIGLMCAVALLILILLIVCFIRRNKGGKYPVKEKEDAHADPEIQPMKEDDGTFGEYSDAEDHKPLKKGSRTPSDRTVKKEDSDDSLVDYGEGVNGQFNEDGSFIGQYSGKKEKEPAEGNESSEAPSPVNAMNSFV",
# NELL1_HUMAN
"MPMDLILVVWFCVCTARTVVGFGMDPDLQMDIVTELDLVNTTLGVAQVSGMHNASKAFLFQDIEREIHAAPHVSEKLIQLFRNKSEFTILATVQQKPSTSGVILSIRELEHSYFELESSGLRDEIRYHYIHNGKPRTEALPYRMADGQWHKVALSVSASHLLLHVDCNRIYERVIDPPDTNLPPGINLWLGQRNQKHGLFKGIIQDGKIIFMPNGYITQCPNLNHTCPTCSDFLSLVQGIMDLQELLAKMTAKLNYAETRLSQLENCHCEKTCQVSGLLYRDQDSWVDGDHCRNCTCKSGAVECRRMSCPPLNCSPDSLPVHIAGQCCKVCRPKCIYGGKVLAEGQRILTKSCRECRGGVLVKITEMCPPLNCSEKDHILPENQCCRVCRGHNFCAEGPKCGENSECKNWNTKATCECKSGYISVQGDSAYCEDIDECAAKMHYCHANTVCVNLPGLYRCDCVPGYIRVDDFSCTEHDECGSGQHNCDENAICTNTVQGHSCTCKPGYVGNGTICRAFCEEGCRYGGTCVAPNKCVCPSGFTGSHCEKDIDECSEGIIECHNHSRCVNLPGWYHCECRSGFHDDGTYSLSGESCIDIDECALRTHTCWNDSACINLAGGFDCLCPSGPSCSGDCPHEGGLKHNGQVWTLKEDRCSVCSCKDGKIFCRRTACDCQNPSADLFCCPECDTRVTSQCLDQNGHKLYRSGDNWTHSCQQCRCLEGEVDCWPLTCPNLSCEYTAILEGECCPRCVSDPCLADNITYDIRKTCLDSYGVSRLSGSVWTMAGSPCTTCKCKNGRVCCSVDFECLQNN",
# KIFA3_HUMAN
"MQGEDARYLKRKVKGGNIDVHPSEKALIVHYEVEATILGEMGDPMLGERKECQKIIRLKSLNANTDITSLARKVVEECKLIHPSKLNEVEQLLYYLQNRRDSLSGKEKKEKSSKPKDPPPFEGMEIDEVANINDMDEYIELLYEDIPDKVRGSALILQLARNPDNLEELLLNETALGALARVLREDWKQSVELATNIIYIFFCFSSFSQFHGLITHYKIGALCMNIIDHELKRHELWQEELSKKKKAVDEDPENQTLRKDYEKTFKKYQGLVVKQEQLLRVALYLLLNLAEDTRTELKMRNKNIVHMLVKALDRDNFELLILVVSFLKKLSIFMENKNDMVEMDIVEKLVKMIPCEHEDLLNITLRLLLNLSFDTGLRNKMVQVGLLPKLTALLGNDNYKQIAMCVLYHISMDDRFKSMFAYTDCIPQLMKMLFECSDERIDLELISFCINLAANKRNVQLICEGNGLKMLMKRALKFKDPLLMKMIRNISQHDGPTKNLFIDYVGDLAAQISNDEEEEFVIECLGTLANLTIPDLDWELVLKEYKLVPYLKDKLKPGAAEDDLVLEVVIMIGTVSMDDSCAALLAKSGIIPALIELLNAQQEDDEFVCQIIYVFYQMVFHQATRDVIIKETQAPAYLIDLMHDKNNEIRKVCDNTLDIIAEYDEEWAKKIQSEKFRWHNSQWLEMVESRQMDESEQYLYGDDRIEPYIHEGDILERPDLFYNSDGLIASEGAISPDFFNDYHLQNGDVVGQHSFPGSLGMDGFGQPVGILGRPATAYGFRPDEPYYYGYGS",
# GHSR_HUMAN
"MWNATPSEEPGFNLTLADLDWDASPGNDSLGDELLQLFPAPLLAGVTATCVALFVVGIAGNLLTMLVVSRFRELRTTTNLYLSSMAFSDLLIFLCMPLDLVRLWQYRPWNFGDLLCKLFQFVSESCTYATVLTITALSVERYFAICFPLRAKVVVTKGRVKLVIFVIWAVAFCSAGPIFVLVGVEHENGTDPWDTNECRPTEFAVRSGLLTVMVWVSSIFFFLPVFCLTVLYSLIGRKLWRRRRGDAVVGASLRDQNHKQTVKMLAVVVFAFILCWLPFHVGRYLFSKSFEPGSLEIAQISQYCNLVSFVLFYLSAAINPILYNIMSKKYRVAVFRLLGFEPFSQRKLSTLKDESSRAWTESSINT",
# SEM4D_HUMAN
"MRMCTPIRGLLMALAVMFGTAMAFAPIPRITWEHREVHLVQFHEPDIYNYSALLLSEDKDTLYIGAREAVFAVNALNISEKQHEVYWKVSEDKKAKCAEKGKSKQTECLNYIRVLQPLSATSLYVCGTNAFQPACDHLNLTSFKFLGKNEDGKGRCPFDPAHSYTSVMVDGELYSGTSYNFLGSEPIISRNSSHSPLRTEYAIPWLNEPSFVFADVIRKSPDSPDGEDDRVYFFFTEVSVEYEFVFRVLIPRIARVCKGDQGGLRTLQKKWTSFLKARLICSRPDSGLVFNVLRDVFVLRSPGLKVPVFYALFTPQLNNVGLSAVCAYNLSTAEEVFSHGKYMQSTTVEQSHTKWVRYNGPVPKPRPGACIDSEARAANYTSSLNLPDKTLQFVKDHPLMDDSVTPIDNRPRLIKKDVNYTQIVVDRTQALDGTVYDVMFVSTDRGALHKAISLEHAVHIIEETQLFQDFEPVQTLLLSSKKGNRFVYAGSNSGVVQAPLAFCGKHGTCEDCVLARDPYCAWSPPTATCVALHQTESPSRGLIQEMSGDASVCPDKSKGSYRQHFFKHGGTAELKCSQKSNLARVFWKFQNGVLKAESPKYGLMGRKNLLIFNLSEGDSGVYQCLSEERVKNKTVFQVVAKHVLEVKVVPKPVVAPTLSVVQTEGSRIATKVLVASTQGSSPPTPAVQATSSGAITLPPKPAPTGTSCEPKIVINTVPQLHSEKTMYLKSSDNRLLMSLFLFFFVLFLCLFFYNCYKGYLPRQCLKFRSALLIGKKKPKSDFCDREQSLKETLVEPGSFSQQNGEHPKPALDTGYETEQDTITSKVPTDREDSQRIDDLSARDKPFDVKCELKFADSDADGD",
# NEO1_HUMAN
"MAAERGARRLLSTPSFWLYCLLLLGRRAPGAAAARSGSAPQSPGASIRTFTPFYFLVEPVDTLSVRGSSVILNCSAYSEPSPKIEWKKDGTFLNLVSDDRRQLLPDGSLFISNVVHSKHNKPDEGYYQCVATVESLGTIISRTAKLIVAGLPRFTSQPEPSSVYAGNNAILNCEVNADLVPFVRWEQNRQPLLLDDRVIKLPSGMLVISNATEGDGGLYRCVVESGGPPKYSDEVELKVLPDPEVISDLVFLKQPSPLVRVIGQDVVLPCVASGLPTPTIKWMKNEEALDTESSERLVLLAGGSLEISDVTEDDAGTYFCIADNGNETIEAQAELTVQAQPEFLKQPTNIYAHESMDIVFECEVTGKPTPTVKWVKNGDMVIPSDYFKIVKEHNLQVLGLVKSDEGFYQCIAENDVGNAQAGAQLIILEHAPATTGPLPSAPRDVVASLVSTRFIKLTWRTPASDPHGDNLTYSVFYTKEGIARERVENTSHPGEMQVTIQNLMPATVYIFRVMAQNKHGSGESSAPLRVETQPEVQLPGPAPNLRAYAASPTSITVTWETPVSGNGEIQNYKLYYMEKGTDKEQDVDVSSHSYTINGLKKYTEYSFRVVAYNKHGPGVSTPDVAVRTLSDVPSAAPQNLSLEVRNSKSIMIHWQPPAPATQNGQITGYKIRYRKASRKSDVTETLVSGTQLSQLIEGLDRGTEYNFRVAALTINGTGPATDWLSAETFESDLDETRVPEVPSSLHVRPLVTSIVVSWTPPENQNIVVRGYAIGYGIGSPHAQTIKVDYKQRYYTIENLDPSSHYVITLKAFNNVGEGIPLYESAVTRPHTDTSEVDLFVINAPYTPVPDPTPMMPPVGVQASILSHDTIRITWADNSLPKHQKITDSRYYTVRWKTNIPANTKYKNANATTLSYLVTGLKPNTLYEFSVMVTKGRRSSTWSMTAHGTTFELVPTSPPKDVTVVSKEGKPKTIIVNWQPPSEANGKITGYIIYYSTDVNAEIHDWVIEPVVGNRLTHQIQELTLDTPYYFKIQARNSKGMGPMSEAVQFRTPKADSSDKMPNDQASGSGGKGSRLPDLGSDYKPPMSGSNSPHGSPTSPLDSNMLLVIIVSVGVITIVVVVIIAVFCTRRTTSHQKKKRAACKSVNGSHKYKGNSKDVKPPDLWIHHERLELKPIDKSPDPNPIMTDTPIPRNSQDITPVDNSMDSNIHQRRNSYRGHESEDSMSTLAGRRGMRPKMMMPFDSQPPQPVISAHPIHSLDNPHHHFHSSSLASPARSHLYHPGSPWPIGTSMSLSDRANSTESVRNTPSTDTMPASSSQTCCTDHQDPEGATSSSYLASSQEEDSGQSLPTAHVRPSHPLKSFAVPAIPPPGPPTYDPALPSTPLLSQQALNHHIHSVKTASIGTLGRSRPPMPVVVPSAPEVQETTRMLEDSESSYEPDELTKEMAHLEGLMKDLNAITTA",
# RAB8B_HUMAN
"MAKTYDYLFKLLLIGDSGVGKTCLLFRFSEDAFNTTFISTIGIDFKIRTIELDGKKIKLQIWDTAGQERFRTITTAYYRGAMGIMLVYDITNEKSFDNIKNWIRNIEEHASSDVERMILGNKCDMNDKRQVSKERGEKLAIDYGIKFLETSAKSSANVEEAFFTLARDIMTKLNRKMNDSNSAGAGGPVKITENRSKKTSFFRCSLL",
# PTPR2_HUMAN
"MGPPLPLLLLLLLLLPPRVLPAAPSSVPRGRQLPGRLGCLLEEGLCGASEACVNDGVFGRCQKVPAMDFYRYEVSPVALQRLRVALQKLSGTGFTWQDDYTQYVMDQELADLPKTYLRRPEASSPARPSKHSVGSERRYSREGGAALANALRRHLPFLEALSQAPASDVLARTHTAQDRPPAEGDDRFSESILTYVAHTSALTYPPGSRTQLREDLLPRTLGQLQPDELSPKVDSGVDRHHLMAALSAYAAQRPPAPPGEGSLEPQYLLRAPSRMPRPLLAPAAPQKWPSPLGDSEDPSSTGDGARIHTLLKDLQRQPAEVRGLSGLELDGMAELMAGLMQGVDHGVARGSPGRAALGESGEQADGPKATLRGDSFPDDGVQDDDDRLYQEVHRLSATLGGLLQDHGSRLLPGALPFARPLDMERKKSEHPESSLSSEEETAGVENVKSQTYSKDLLGQQPHSEPGAAAFGELQNQMPGPSKEEQSLPAGAQEALSDGLQLEVQPSEEEARGYIVTDRDPLRPEEGRRLVEDVARLLQVPSSAFADVEVLGPAVTFKVSANVQNVTTEDVEKATVDNKDKLEETSGLKILQTGVGSKSKLKFLPPQAEQEDSTKFIALTLVSLACILGVLLASGLIYCLRHSSQHRLKEKLSGLGGDPGADATAAYQELCRQRMATRPPDRPEGPHTSRISSVSSQFSDGPIPSPSARSSASSWSEEPVQSNMDISTGHMILSYMEDHLKNKNRLEKEWEALCAYQAEPNSSFVAQREENVPKNRSLAVLTYDHSRVLLKAENSHSHSDYINASPIMDHDPRNPAYIATQGPLPATVADFWQMVWESGCVVIVMLTPLAENGVRQCYHYWPDEGSNLYHIYEVNLVSEHIWCEDFLVRSFYLKNLQTNETRTVTQFHFLSWYDRGVPSSSRSLLDFRRKVNKCYRGRSCPIIVHCSDGAGRSGTYVLIDMVLNKMAKGAKEIDIAATLEHLRDQRPGMVQTKEQFEFALTAVAEEVNAILKALPQ",
# ARHG2_HUMAN
"MSRIESLTRARIDRSRELASKTREKEKMKEAKDARYTNGHLFTTISVSGMTMCYACNKSITAKEALICPTCNVTIHNRCKDTLANCTKVKQKQQKAALLKNNTALQSVSLRSKTTIRERPSSAIYPSDSFRQSLLGSRRGRSSLSLAKSVSTTNIAGHFNDESPLGLRRILSQSTDSLNMRNRTLSVESLIDEAEVIYSELMSDFEMDEKDFAADSWSLAVDSSFLQQHKKEVMKQQDVIYELIQTELHHVRTLKIMTRLFRTGMLEELHLEPGVVQGLFPCVDELSDIHTRFLSQLLERRRQALCPGSTRNFVIHRLGDLLISQFSGPSAEQMCKTYSEFCSRHSKALKLYKELYARDKRFQQFIRKVTRPAVLKRHGVQECILLVTQRITKYPLLISRILQHSHGIEEERQDLTTALGLVKELLSNVDEGIYQLEKGARLQEIYNRMDPRAQTPVPGKGPFGREELLRRKLIHDGCLLWKTATGRFKDVLVLLMTDVLVFLQEKDQKYIFPTLDKPSVVSLQNLIVRDIANQEKGMFLISAAPPEMYEVHTASRDDRSTWIRVIQQSVRTCPSREDFPLIETEDEAYLRRIKMELQQKDRALVELLREKVGLFAEMTHFQAEEDGGSGMALPTLPRGLFRSESLESPRGERLLQDAIREVEGLKDLLVGPGVELLLTPREPALPLEPDSGGNTSPGVTANGEARTFNGSIELCRADSDSSQRDRNGNQLRSPQEEALQRLVNLYGLLHGLQAAVAQQDTLMEARFPEGPERREKLCRANSRDGEAGRAGAAPVAPEKQATELALLQRQHALLQEELRRCRRLGEERATEAGSLEARLRESEQARALLEREAEEARRQLAALGQTEPLPAEAPWARRPVDPRRRSLPAGDALYLSFNPPQPSRGTDRLDLPVTTRSVHRNFEDRERQELGSPEERLQDSSDPDTGSEEEGSSRLSPPHSPRDFTRMQDIPEETESRDGEAVASES",
# DVL3_HUMAN
"MGETKIIYHLDGQETPYLVKLPLPAERVTLADFKGVLQRPSYKFFFKSMDDDFGVVKEEISDDNAKLPCFNGRVVSWLVSAEGSHPDPAPFCADNPSELPPPMERTGGIGDSRPPSFHPHAGGGSQENLDNDTETDSLVSAQRERPRRRDGPEHATRLNGTAKGERRREPGGYDSSSTLMSSELETTSFFDSDEDDSTSRFSSSTEQSSASRLMRRHKRRRRKQKVSRIERSSSFSSITDSTMSLNIITVTLNMEKYNFLGISIVGQSNERGDGGIYIGSIMKGGAVAADGRIEPGDMLLQVNEINFENMSNDDAVRVLREIVHKPGPITLTVAKCWDPSPRGCFTLPRSEPIRPIDPAAWVSHTAAMTGTFPAYGMSPSLSTITSTSSSITSSIPDTERLDDFHLSIHSDMAAIVKAMASPESGLEVRDRMWLKITIPNAFIGSDVVDWLYHNVEGFTDRREARKYASNLLKAGFIRHTVNKITFSEQCYYIFGDLCGNMANLSLHDHDGSSGASDQDTLAPLPHPGAAPWPMAFPYQYPPPPHPYNPHPGFPELGYSYGGGSASSQHSEGSRSSGSNRSGSDRRKEKDPKAGDSKSGGSGSESDHTTRSSLRGPRERAPSERSGPAASEHSHRSHHSLASSLRSHHTHPSYGPPGVPPLYGPPMLMMPPPPAAMGPPGAPPGRDLASVPPELTASRQSFRMAMGNPSEFFVDVM",
# USP9X_HUMAN
"MTATTRGSPVGGNDNQGQAPDGQSQPPLQQNQTSSPDSSNENSPATPPDEQGQGDAPPQLEDEEPAFPHTDLAKLDDMINRPRWVVPVLPKGELEVLLEAAIDLSKKGLDVKSEACQRFFRDGLTISFTKILTDEAVSGWKFEIHRCIINNTHRLVELCVAKLSQDWFPLLELLAMALNPHCKFHIYNGTRPCESVSSSVQLPEDELFARSPDPRSPKGWLVDLLNKFGTLNGFQILHDRFINGSALNVQIIAALIKPFGQCYEFLTLHTVKKYFLPIIEMVPQFLENLTDEELKKEAKNEAKNDALSMIIKSLKNLASRVPGQEETVKNLEIFRLKMILRLLQISSFNGKMNALNEVNKVISSVSYYTHRHGNPEEEEWLTAERMAEWIQQNNILSIVLRDSLHQPQYVEKLEKILRFVIKEKALTLQDLDNIWAAQAGKHEAIVKNVHDLLAKLAWDFSPEQLDHLFDCFKASWTNASKKQREKLLELIRRLAEDDKDGVMAHKVLNLLWNLAHSDDVPVDIMDLALSAHIKILDYSCSQDRDTQKIQWIDRFIEELRTNDKWVIPALKQIREICSLFGEAPQNLSQTQRSPHVFYRHDLINQLQHNHALVTLVAENLATYMESMRLYARDHEDYDPQTVRLGSRYSHVQEVQERLNFLRFLLKDGQLWLCAPQAKQIWKCLAENAVYLCDREACFKWYSKLMGDEPDLDPDINKDFFESNVLQLDPSLLTENGMKCFERFFKAVNCREGKLVAKRRAYMMDDLELIGLDYLWRVVIQSNDDIASRAIDLLKEIYTNLGPRLQVNQVVIHEDFIQSCFDRLKASYDTLCVLDGDKDSVNCARQEAVRMVRVLTVLREYINECDSDYHEERTILPMSRAFRGKHLSFVVRFPNQGRQVDDLEVWSHTNDTIGSVRRCILNRIKANVAHTKIELFVGGELIDPADDRKLIGQLNLKDKSLITAKLTQISSNMPSSPDSSSDSSTGSPGNHGNHYSDGPNPEVESCLPGVIMSLHPRYISFLWQVADLGSSLNMPPLRDGARVLMKLMPPDSTTIEKLRAICLDHAKLGESSLSPSLDSLFFGPSASQVLYLTEVVYALLMPAGAPLADDSSDFQFHFLKSGGLPLVLSMLTRNNFLPNADMETRRGAYLNALKIAKLLLTAIGYGHVRAVAEACQPGVEGVNPMTQINQVTHDQAVVLQSALQSIPNPSSECMLRNVSVRLAQQISDEASRYMPDICVIRAIQKIIWASGCGSLQLVFSPNEEITKIYEKTNAGNEPDLEDEQVCCEALEVMTLCFALIPTALDALSKEKAWQTFIIDLLLHCHSKTVRQVAQEQFFLMCTRCCMGHRPLLFFITLLFTVLGSTARERAKHSGDYFTLLRHLLNYAYNSNINVPNAEVLLNNEIDWLKRIRDDVKRTGETGIEETILEGHLGVTKELLAFQTSEKKFHIGCEKGGANLIKELIDDFIFPASNVYLQYMRNGELPAEQAIPVCGSPPTINAGFELLVALAVGCVRNLKQIVDSLTEMYYIGTAITTCEALTEWEYLPPVGPRPPKGFVGLKNAGATCYMNSVIQQLYMIPSIRNGILAIEGTGSDVDDDMSGDEKQDNESNVDPRDDVFGYPQQFEDKPALSKTEDRKEYNIGVLRHLQVIFGHLAASRLQYYVPRGFWKQFRLWGEPVNLREQHDALEFFNSLVDSLDEALKALGHPAMLSKVLGGSFADQKICQGCPHRYECEESFTTLNVDIRNHQNLLDSLEQYVKGDLLEGANAYHCEKCNKKVDTVKRLLIKKLPPVLAIQLKRFDYDWERECAIKFNDYFEFPRELDMEPYTVAGVAKLEGDNVNPESQLIQQSEQSESETAGSTKYRLVGVLVHSGQASGGHYYSYIIQRNGGDGERNRWYKFDDGDVTECKMDDDEEMKNQCFGGEYMGEVFDHMMKRMSYRRQKRWWNAYILFYERMDTIDQDDELIRYISELAITTRPHQIIMPSAIERSVRKQNVQFMHNRMQYSMEYFQFMKKLLTCNGVYLNPPPGQDHLLPEAEEITMISIQLAARFLFTTGFHTKKVVRGSASDWYDALCILLRHSKNVRFWFAHNVLFNVSNRFSEYLLECPSAEVRGAFAKLIVFIAHFSLQDGPCPSPFASPGPSSQAYDNLSLSDHLLRAVLNLLRREVSEHGRHLQQYFNLFVMYANLGVAEKTQLLKLSVPATFMLVSLDEGPGPPIKYQYAELGKLYSVVSQLIRCCNVSSRMQSSINGNPPLPNPFGDPNLSQPIMPIQQNVADILFVRTSYVKKIIEDCSNSEETVKLLRFCCWENPQFSSTVLSELLWQVAYSYTYELRPYLDLLLQILLIEDSWQTHRIHNALKGIPDDRDGLFDTIQRSKNHYQKRAYQCIKCMVALFSNCPVAYQILQGNGDLKRKWTWAVEWLGDELERRPYTGNPQYTYNNWSPPVQSNETSNGYFLERSHSARMTLAKACELCPEEEPDDQDAPDEHESPPPEDAPLYPHSPGSQYQQNNHVHGQPYTGPAAHHMNNPQRTGQRAQENYEGSEEVSPPQTKDQ",
# VPP1_HUMAN
"MGELFRSEEMTLAQLFLQSEAAYCCVSELGELGKVQFRDLNPDVNVFQRKFVNEVRRCEEMDRKLRFVEKEIRKANIPIMDTGENPEVPFPRDMIDLEANFEKIENELKEINTNQEALKRNFLELTELKFILRKTQQFFDEMADPDLLEESSSLLEPSEMGRGTPLRLGFVAGVINRERIPTFERMLWRVCRGNVFLRQAEIENPLEDPVTGDYVHKSVFIIFFQGDQLKNRVKKICEGFRASLYPCPETPQERKEMASGVNTRIDDLQMVLNQTEDHRQRVLQAAAKNIRVWFIKVRKMKAIYHTLNLCNIDVTQKCLIAEVWCPVTDLDSIQFALRRGTEHSGSTVPSILNRMQTNQTPPTYNKTNKFTYGFQNIVDAYGIGTYREINPAPYTIITFPFLFAVMFGDFGHGILMTLFAVWMVLRESRILSQKNENEMFSTVFSGRYIILLMGVFSMYTGLIYNDCFSKSLNIFGSSWSVRPMFTYNWTEETLRGNPVLQLNPALPGVFGGPYPFGIDPIWNIATNKLTFLNSFKMKMSVILGIIHMLFGVSLSLFNHIYFKKPLNIYFGFIPEIIFMTSLFGYLVILIFYKWTAYDAHTSENAPSLLIHFINMFLFSYPESGYSMLYSGQKGIQCFLVVVALLCVPWMLLFKPLVLRRQYLRRKHLGTLNFGGIRVGNGPTEEDAEIIQHDQLSTHSEDADEPSEDEVFDFGDTMVHQAIHTIEYCLGCISNTASYLRLWALSLAHAQLSEVLWTMVIHIGLSVKSLAGGLVLFFFFTAFATLTVAILLIMEGLSAFLHALRLHWVEFQNKFYSGTGFKFLPFSFEHIREGKFEE",
# RNF34_HUMAN
"MKAGATSMWASCCGLLNEVMGTGAVRGQQSAFAGATGPFRFTPNPEFSTYPPAATEGPNIVCKACGLSFSVFRKKHVCCDCKKDFCSVCSVLQENLRRCSTCHLLQETAFQRPQLMRLKVKDLRQYLILRNIPIDTCREKEDLVDLVLCHHGLGSEDDMDTSSLNSSRSQTSSFFTRSFFSNYTAPSATMSSFQGELMDGDQTSRSGVPAQVQSEITSANTEDDDDDDDEDDDDEEENAEDRNPGLSKERVRASLSDLSSLDDVEGMSVRQLKEILARNFVNYSGCCEKWELVEKVNRLYKENEENQKSYGERLQLQDEEDDSLCRICMDAVIDCVLLECGHMVTCTKCGKRMSECPICRQYVVRAVHVFKS",
# MAL2_HUMAN
"MSAGGASVPPPPNPAVSFPPPRVTLPAGPDILRTYSGAFVCLEILFGGLVWILVASSNVPLPLLQGWVMFVSVTAFFFSLLFLGMFLSGMVAQIDANWNFLDFAYHFTVFVFYFGAFLLEAAATSLHDLHCNTTITGQPLLSDNQYNINVAASIFAFMTTACYGCSLGLALRRWRP",
# IGSF8_HUMAN
"MGALRPTLLPPSLPLLLLLMLGMGCWAREVLVPEGPLYRVAGTAVSISCNVTGYEGPAQQNFEWFLYRPEAPDTALGIVSTKDTQFSYAVFKSRVVAGEVQVQRLQGDAVVLKIARLQAQDAGIYECHTPSTDTRYLGSYSGKVELRVLPDVLQVSAAPPGPRGRQAPTSPPRMTVHEGQELALGCLARTSTQKHTHLAVSFGRSVPEAPVGRSTLQEVVGIRSDLAVEAGAPYAERLAAGELRLGKEGTDRYRMVVGGAQAGDAGTYHCTAAEWIQDPDGSWAQIAEKRAVLAHVDVQTLSSQLAVTVGPGERRIGPGEPLELLCNVSGALPPAGRHAAYSVGWEMAPAGAPGPGRLVAQLDTEGVGSLGPGYEGRHIAMEKVASRTYRLRLEAARPGDAGTYRCLAKAYVRGSGTRLREAASARSRPLPVHVREEGVVLEAVAWLAGGTVYRGETASLLCNISVRGGPPGLRLAASWWVERPEDGELSSVPAQLVGGVGQDGVAELGVRPGGGPVSVELVGPRSHRLRLHSLGPEDEGVYHCAPSAWVQHADYSWYQAGSARSGPVTVYPYMHALDTLFVPLLVGTGVALVTGATVLGTITCCFMKRLRKR",
# MRTFA_HUMAN
"MPPLKSPAAFHEQRRSLERARTEDYLKRKIRSRPERSELVRMHILEETSAEPSLQAKQLKLKRARLADDLNEKIAQRPGPMELVEKNILPVESSLKEAIIVGQVNYPKVADSSSFDEDSSDALSPEQPASHESQGSVPSPLEARVSEPLLSATSASPTQVVSQLPMGRDSREMLFLAEQPPLPPPPLLPPSLTNGTTIPTAKSTPTLIKQSQPKSASEKSQRSKKAKELKPKVKKLKYHQYIPPDQKQDRGAPPMDSSYAKILQQQQLFLQLQILNQQQQQHHNYQAILPAPPKSAGEALGSSGTPPVRSLSTTNSSSSSGAPGPCGLARQNSTSLTGKPGALPANLDDMKVAELKQELKLRSLPVSGTKTELIERLRAYQDQISPVPGAPKAPAATSILHKAGEVVVAFPAARLSTGPALVAAGLAPAEVVVATVASSGVVKFGSTGSTPPVSPTPSERSLLSTGDENSTPGDTFGEMVTSPLTQLTLQASPLQILVKEEGPRAGSCCLSPGGRAELEGRDKDQMLQEKDKQIEALTRMLRQKQQLVERLKLQLEQEKRAQQPAPAPAPLGTPVKQENSFSSCQLSQQPLGPAHPFNPSLAAPATNHIDPCAVAPGPPSVVVKQEALQPEPEPVPAPQLLLGPQGPSLIKGVAPPTLITDSTGTHLVLTVTNKNADSPGLSSGSPQQPSSQPGSPAPAPSAQMDLEHPLQPLFGTPTSLLKKEPPGYEEAMSQQPKQQENGSSSQQMDDLFDILIQSGEISADFKEPPSLPGKEKPSPKTVCGSPLAAQPSPSAELPQAAPPPPGSPSLPGRLEDFLESSTGLPLLTSGHDGPEPLSLIDDLHSQMLSSTAILDHPPSPMDTSELHFVPEPSSTMGLDLADGHLDSMDWLELSSGGPVLSLAPLSTTAPSLFSTDFLDGHDLQLHWDSCL",
# CPNE4_HUMAN
"MKKMSNIYESAANTLGIFNSPCLTKVELRVACKGISDRDALSKPDPCVILKMQSHGQWFEVDRTEVIRTCINPVYSKLFTVDFYFEEVQRLRFEVHDISSNHNGLKEADFLGGMECTLGQIVSQRKLSKSLLKHGNTAGKSSITVIAEELSGNDDYVELAFNARKLDDKDFFSKSDPFLEIFRMNDDATQQLVHRTEVVMNNLSPAWKSFKVSVNSLCSGDPDRRLKCIVWDWDSNGKHDFIGEFTSTFKEMRGAMEGKQVQWECINPKYKAKKKNYKNSGTVILNLCKIHKMHSFLDYIMGGCQIQFTVAIDFTASNGDPRNSCSLHYIHPYQPNEYLKALVAVGEICQDYDSDKMFPAFGFGARIPPEYTVSHDFAINFNEDNPECAGIQGVVEAYQSCLPKLQLYGPTNIAPIIQKVAKSASEETNTKEASQYFILLILTDGVITDMADTREAIVHASHLPMSVIIVGVGNADFSDMQMLDGDDGILRSPKGEPVLRDIVQFVPFRNFKHASPAALAKSVLAEVPNQVVDYYNGKGIKPKCSSEMYESSRTLAP",
# TM230_HUMAN
"MMPSRTNLATGIPSSKVKYSRLSSTDDGYIDLQFKKTPPKIPYKAIALATVLFLIGAFLIIIGSLLLSGYISKGGADRAVPVLIIGILVFLPGFYHLRIAYYASKGYRGYSYDDIPDFDD",
# EXOC4_HUMAN
"MAAEAAGGKYRSTVSKSKDPSGLLISVIRTLSTSDDVEDRENEKGRLEEAYEKCDRDLDELIVQHYTELTTAIRTYQSITERITNSRNKIKQVKENLLSCKMLLHCKRDELRKLWIEGIEHKHVLNLLDEIENIKQVPQKLEQCMASKHYLSATDMLVSAVESLEGPLLQVEGLSDLRLELHSKKMNLHLVLIDELHRHLYIKSTSRVVQRNKEKGKISSLVKDASVPLIDVTNLPTPRKFLDTSHYSTAGSSSVREINLQDIKEDLELDPEENSTLFMGILIKGLAKLKKIPETVKAIIERLEQELKQIVKRSTTQVADSGYQRGENVTVENQPRLLLELLELLFDKFNAVAAAHSVVLGYLQDTVVTPLTQQEDIKLYDMADVWVKIQDVLQMLLTEYLDMKNTRTASEPSAQLSYASTGREFAAFFAKKKPQRPKNSLFKFESSSHAISMSAYLREQRRELYSRSGELQGGPDDNLIEGGGTKFVCKPGARNITVIFHPLLRFIQEIEHALGLGPAKQCPLREFLTVYIKNIFLNQVLAEINKEIEGVTKTSDPLKILANADTMKVLGVQRPLLQSTIIVEKTVQDLLNLMHDLSAYSDQFLNMVCVKLQEYKDTCTAAYRGIVQSEEKLVISASWAKDDDISRLLKSLPNWMNMAQPKQLRPKREEEEDFIRAAFGKESEVLIGNLGDKLIPPQDILRDVSDLKALANMHESLEWLASRTKSAFSNLSTSQMLSPAQDSHTNTDLPPVSEQIMQTLSELAKSFQDMADRCLLVLHLEVRVHCFHYLIPLAKEGNYAIVANVESMDYDPLVVKLNKDISAIEEAMSASLQQHKFQYIFEGLGHLISCILINGAQYFRRISESGIKKMCRNIFVLQQNLTNITMSREADLDFARQYYEMLYNTADELLNLVVDQGVKYTELEYIHALTLLHRSQTGVGELTTQNTRLQRLKEIICEQAAIKQATKDKKITTV",
# FUBP1_HUMAN
"MADYSTVPPPSSGSAGGGGGGGGGGGVNDAFKDALQRARQIAAKIGGDAGTSLNSNDYGYGGQKRPLEDGDQPDAKKVAPQNDSFGTQLPPMHQQQSRSVMTEEYKVPDGMVGFIIGRGGEQISRIQQESGCKIQIAPDSGGLPERSCMLTGTPESVQSAKRLLDQIVEKGRPAPGFHHGDGPGNAVQEIMIPASKAGLVIGKGGETIKQLQERAGVKMVMIQDGPQNTGADKPLRITGDPYKVQQAKEMVLELIRDQGGFREVRNEYGSRIGGNEGIDVPIPRFAVGIVIGRNGEMIKKIQNDAGVRIQFKPDDGTTPERIAQITGPPDRCQHAAEIITDLLRSVQAGNPGGPGPGGRGRGRGQGNWNMGPPGGLQEFNFIVPTGKTGLIIGKGGETIKSISQQSGARIELQRNPPPNADPNMKLFTIRGTPQQIDYARQLIEEKIGGPVNPLGPPVPHGPHGVPGPHGPPGPPGPGTPMGPYNPAPYNPGPPGPAPHGPPAPYAPQGWGNAYPHWQQQAPPDPAKAGTDPNSAAWAAYYAHYYQQQAQPPPAAPAGAPTTTQTNGQGDQQNPAPAGQVDYTKAWEEYYKKMGQAVPAPTGAPPGGQPDYSAAWAEYYRQQAAYYAQTSPQGMPQHPPAPQGQ",
# VTI1A_HUMAN
"MSSDFEGYEQDFAVLTAEITSKIARVPRLPPDEKKQMVANVEKQLEEAKELLEQMDLEVREIPPQSRGMYSNRMRSYKQEMGKLETDFKRSRIAYSDEVRNELLGDDGNSSENQRAHLLDNTERLERSSRRLEAGYQIAVETEQIGQEMLENLSHDREKIQRARERLRETDANLGKSSRILTGMLRRIIQNRILLVILGIIVVITILMAITFSVRRH",
# MIB2_HUMAN
"MDPDPQAGVQVGMRVVRGVDWKWGQQDGGEGGVGTVVELGRHGSPSTPDRTVVVQWDQGTRTNYRAGYQGAHDLLLYDNAQIGVRHPNIICDCCKKHGLRGMRWKCRVCLDYDLCTQCYMHNKHELAHAFDRYETAHSRPVTLSPRQGLPRIPLRGIFQGAKVVRGPDWEWGSQDGGEGKPGRVVDIRGWDVETGRSVASVTWADGTTNVYRVGHKGKVDLKCVGEAAGGFYYKDHLPRLGKPAELQRRVSADSQPFQHGDKVKCLLDTDVLREMQEGHGGWNPRMAEFIGQTGTVHRITDRGDVRVQFNHETRWTFHPGALTKHHSFWVGDVVRVIGDLDTVKRLQAGHGEWTDDMAPALGRVGKVVKVFGDGNLRVAVAGQRWTFSPSCLVAYRPEEDANLDVAERARENKSSLSVALDKLRAQKSDPEHPGRLVVEVALGNAARALDLLRRRPEQVDTKNQGRTALQVAAYLGQVELIRLLLQARAGVDLPDDEGNTALHYAALGNQPEATRVLLSAGCRADAINSTQSTALHVAVQRGFLEVVRALCERGCDVNLPDAHSDTPLHSAISAGTGASGIVEVLTEVPNIDVTATNSQGFTLLHHASLKGHALAVRKILARARQLVDAKKEDGFTALHLAALNNHREVAQILIREGRCDVNVRNRKLQSPLHLAVQQAHVGLVPLLVDAGCSVNAEDEEGDTALHVALQRHQLLPLVADGAGGDPGPLQLLSRLQASGLPGSAELTVGAAVACFLALEGADVSYTNHRGRSPLDLAAEGRVLKALQGCAQRFRERQAGGGAAPGPRQTLGTPNTVTNLHVGAAPGPEAAECLVCSELALLVLFSPCQHRTVCEECARRMKKCIRCQVVVSKKLRPDGSEVASAAPAPGPPRQLVEELQSRYRQMEERITCPICIDSHIRLVFQCGHGACAPCGSALSACPICRQPIRDRIQIFV",
# SH3K1_HUMAN
"MVEAIVEFDYQAQHDDELTISVGEIITNIRKEDGGWWEGQINGRRGLFPDNFVREIKKEMKKDPLTNKAPEKPLHEVPSGNSLLSSETILRTNKRGERRRRRCQVAFSYLPQNDDELELKVGDIIEVVGEVEEGWWEGVLNGKTGMFPSNFIKELSGESDELGISQDEQLSKSSLRETTGSESDGGDSSSTKSEGANGTVATAAIQPKKVKGVGFGDIFKDKPIKLRPRSIEVENDFLPVEKTIGKKLPATTATPDSSKTEMDSRTKSKDYCKVIFPYEAQNDDELTIKEGDIVTLINKDCIDVGWWEGELNGRRGVFPDNFVKLLPPDFEKEGNRPKKPPPPSAPVIKQGAGTTERKHEIKKIPPERPEMLPNRTEEKERPEREPKLDLQKPSVPAIPPKKPRPPKTNSLSRPGALPPRRPERPVGPLTHTRGDSPKIDLAGSSLSGILDKDLSDRSNDIDLEGFDSVVSSTEKLSHPTTSRPKATGRRPPSQSLTSSSLSSPDIFDSPSPEEDKEEHISLAHRGVDASKKTSKTVTISQVSDNKASLPPKPGTMAAGGGGPAPLSSAAPSPLSSSLGTAGHRANSPSLFGTEGKPKMEPAASSQAAVEELRTQVRELRSIIETMKDQQKREIKQLLSELDEEKKIRLRLQMEVNDIKKALQSK",
# APH1A_HUMAN
"MGAAVFFGCTFVAFGPAFALFLITVAGDPLRVIILVAGAFFWLVSLLLASVVWFILVHVTDRSDARLQYGLLIFGAAVSVLLQEVFRFAYYKLLKKADEGLASLSEDGRSPISIRQMAYVSGLSFGIISGVFSVINILADALGPGVVGIHGDSPYYFLTSAFLTAAIILLHTFWGVVFFDACERRRYWALGLVVGSHLLTSGLTFLNPWYEASLLPIYAVTVSMGLWAFITAGGSLRSIQRSLLCRRQEDSRVMVYSALRIPPED",
# ARL8A_HUMAN
"MIALFNKLLDWFKALFWKEEMELTLVGLQYSGKTTFVNVIASGQFNEDMIPTVGFNMRKITKGNVTIKLWDIGGQPRFRSMWERYCRGVSAIVYMVDAADQEKIEASKNELHNLLDKPQLQGIPVLVLGNKRDLPGALDEKELIEKMNLSAIQDREICCYSISCKEKDNIDITLQWLIQHSKSRRS",
# DOC10_HUMAN
"MAGERTRRFTRSLLRPGQAAELRHSAASAAAVAVSSRQQQRQEKPRLLEPLDYETVIEELEKTYRNDPLQDLLFFPSDDFSAATVSWDIRTLYSTVPEDAEHKAENLLVKEACKFYSSQWHVVNYKYEQYSGDIRQLPRAEYKPEKLPSHSFEIDHEDADKDEDTTSHSSSKGGGGAGGTGVFKSGWLYKGNFNSTVNNTVTVRSFKKRYFQLTQLPDNSYIMNFYKDEKISKEPKGCIFLDSCTGVVQNNRLRKYAFELKMNDLTYFVLAAETESDMDEWIHTLNRILQISPEGPLQGRRSTELTDLGLDSLDNSVTCECTPEETDSSENNLHADFAKYLTETEDTVKTTRNMERLNLFSLDPDIDTLKLQKKDLLEPESVIKPFEEKAAKRIMIICKALNSNLQGCVTENENDPITNIEPFFVSVALYDLRDSRKISADFHVDLNHAAVRQMLLGASVALENGNIDTITPRQSEEPHIKGLPEEWLKFPKQAVFSVSNPHSEIVLVAKIEKVLMGNIASGAEPYIKNPDSNKYAQKILKSNRQFCSKLGKYRMPFAWAVRSVFKDNQGNVDRDSRFSPLFRQESSKISTEDLVKLVSDYRRADRISKMQTIPGSLDIAVDNVPLEHPNCVTSSFIPVKPFNMMAQTEPTVEVEEFVYDSTKYCRPYRVYKNQIYIYPKHLKYDSQKCFNKARNITVCIEFKNSDEESAKPLKCIYGKPGGPLFTSAAYTAVLHHSQNPDFSDEVKIELPTQLHEKHHILFSFYHVTCDINAKANAKKKEALETSVGYAWLPLMKHDQIASQEYNIPIATSLPPNYLSFQDSASGKHGGSDIKWVDGGKPLFKVSTFVVSTVNTQDPHVNAFFQECQKREKDMSQSPTSNFIRSCKNLLNVEKIHAIMSFLPIILNQLFKVLVQNEEDEITTTVTRVLTDIVAKCHEEQLDHSVQSYIKFVFKTRACKERTVHEELAKNVTGLLKSNDSTTVKHVLKHSWFFFAIILKSMAQHLIDTNKIQLPRPQRFPESYQNELDNLVMVLSDHVIWKYKDALEETRRANHSVARFLKRCFTFMDRGYVFKMVNNYISMFSSGDLKTLCQYKFDFLQEVCQHEHFIPLCLPIRSANIPDPLTPSESTQELHASDMPEYSVTNEFCRKHFLIGILLREVGFALQEDQDVRHLALAVLKNLMAKHSFDDRYREPRKQAQIASLYMPLYGMLLDNMPRIYLKDLYPFTVNTSNQGSRDDLSTNGGFQSQTAIKHANSVDTSFSKDVLNSIAAFSSIAISTVNHADSRASLASLDSNPSTNEKSSEKTDNCEKIPRPLSLIGSTLRFDKLDQAETRSLLMCFLHIMKTISYETLIAYWQRAPSPEVSDFFSILDVCLQNFRYLGKRNIIRKIAAAFKFVQSTQNNGTLKGSNPSCQTSGLLSQWMHSTSSHEGHKQHRSQTLPIIRGKNALSNPKLLQMLDNTMTSNSNEIDIVHHVDTEANIATEVCLTILDLLSLFTQTHQRQLQQCDCQNSLMKRVFDTYMLFFQVNQSATALKHVFASLRLFVCKFPSAFFQGPADLCGSFCYEVLKCCNHRSRSTQTEASALLYFFMRKNFEFNKQKSIVRSHLQLIKAVSQLIADAGIGGSRFQHSLAITNNFANGDKQMKNSNFPAEVKDLTKRIRTVLMATAQMKEHEKDPEMLVDLQYSLANSYASTPELRRTWLESMAKIHARNGDLSEAAMCYIHIAALIAEYLKRKGYWKVEKICTASLLSEDTHPCDSNSLLTTPSGGSMFSMGWPAFLSITPNIKEEGAMKEDSGMQDTPYNENILVEQLYMCVEFLWKSERYELIADVNKPIIAVFEKQRDFKKLSDLYYDIHRSYLKVAEVVNSEKRLFGRYYRVAFYGQGFFEEEEGKEYIYKEPKLTGLSEISQRLLKLYADKFGADNVKIIQDSNKVNPKDLDPKYAYIQVTYVTPFFEEKEIEDRKTDFEMHHNINRFVFETPFTLSGKKHGGVAEQCKRRTILTTSHLFPYVKKRIQVISQSSTELNPIEVAIDEMSKKVSELNQLCTMEEVDMIRLQLKLQGSVSVKVNAGPMAYARAFLEETNAKKYPDNQVKLLKEIFRQFADACGQALDVNERLIKEDQLEYQEELRSHYKDMLSELSTVMNEQITGRDDLSKRGVDQTCTRVISKATPALPTVSISSSAEV",
# EVI5L_HUMAN
"MASPTLSPDSSSQEALSAPTCSPTSDSENLSPDELELLAKLEEQNRLLEADSKSMRSMNGSRRNSGSSLVSSSSASSNLSHLEEDTWILWGRIANEWEEWRRRKEKLLKELIRKGIPHHFRAIVWQLLCSATDMPVKNQYSELLKMSSPCEKLIRRDIARTYPEHEFFKGQDSLGQEVLFNVMKAYSLVDREVGYCQGSAFIVGLLLMQMPEEEAFCVFVRLMQEYRLRELFKPSMAELGLCIYQFEYMLQEQLPDLNTHFRSQSFHTSMYASSWFLTLFLTTFPLPVATRVFDIFMYEGLEIVFRVGLALLQVNQAELMQLDMEGMSQYFQRVIPHQFDSCPDKLVLKAYQVKYNPKKMKRLEKEYAAMKSKEMEEQIEIKRLRTENRLLKQRIETLEKGQVTRAQEAEENYVIKRELAVVRQQCSSAAEDLQKAQSTIRQLQEQQENPRLTEDFVSHLETELEQSRLRETETLGALREMQDKVLDMEKRNSSLPDENNVAQLQEELKALKVREGQAVASTRELKLQLQELSDTWQAHLARGGRWKESPRKLVVGELQDELMSVRLREAQALAEGRELRQRVVELETQDHIHRNLLNRVEAERAALQEKLQYLAAQNKGLQTQLSESRRKQAEAECKSKEEVMAVRLREADSMAAVAEMRQRIAELEIQREEGRIQGQLNHSDSSQYIRELKDQIEELKAEVRLLKGPPPFEDPLAFDGLSLARHLDEDSLPSSDEELLGVGVGAALQDALYPLSPRDARFFRRLERPAKDSEGSSDSDADELAAPYSQGLDN",
# AP2M1_HUMAN
"MIGGLFIYNHKGEVLISRVYRDDIGRNAVDAFRVNVIHARQQVRSPVTNIARTSFFHVKRSNIWLAAVTKQNVNAAMVFEFLYKMCDVMAAYFGKISEENIKNNFVLIYELLDEILDFGYPQNSETGALKTFITQQGIKSQHQTKEEQSQITSQVTGQIGWRREGIKYRRNELFLDVLESVNLLMSPQGQVLSAHVSGRVVMKSYLSGMPECKFGMNDKIVIEKQGKGTADETSKSGKQSIAIDDCTFHQCVRLSKFDSERSISFIPPDGEFELMRYRTTKDIILPFRVIPLVREVGRTKLEVKVVIKSNFKPSLLAQKIEVRIPTPLNTSGVQVICMKGKAKYKASENAIVWKIKRMAGMKESQISAEIELLPTNDKKKWARPPISMNFEVPFAPSGLKVRYLKVFEPKLNYSDHDVIKWVRYIGRSGIYETRC",
# NTNG2_HUMAN
"MLHLLALFLHCLPLASGDYDICKSWVTTDEGPTWEFYACQPKVMRLKDYVKVKVEPSGITCGDPPERFCSHENPYLCSNECDASNPDLAHPPRLMFDKEEEGLATYWQSITWSRYPSPLEANITLSWNKTVELTDDVVMTFEYGRPTVMVLEKSLDNGRTWQPYQFYAEDCMEAFGMSARRARDMSSSSAHRVLCTEEYSRWAGSKKEKHVRFEVRDRFAIFAGPDLRNMDNLYTRLESAKGLKEFFTLTDLRMRLLRPALGGTYVQRENLYKYFYAISNIEVIGRCKCNLHANLCSMREGSLQCECEHNTTGPDCGKCKKNFRTRSWRAGSYLPLPHGSPNACATAGSFGNCECYGHSNRCSYIDFLNVVTCVSCKHNTRGQHCQHCRLGYYRNGSAELDDENVCIECNCNQIGSVHDRCNETGFCECREGAAGPKCDDCLPTHYWRQGCYPNVCDDDQLLCQNGGTCLQNQRCACPRGYTGVRCEQPRCDPADDDGGLDCDRAPGAAPRPATLLGCLLLLGLAARLGR",
# KCD12_HUMAN
"MALADSTRGLPNGGGGGGGSGSSSSSAEPPLFPDIVELNVGGQVYVTRRCTVVSVPDSLLWRMFTQQQPQELARDSKGRFFLDRDGFLFRYILDYLRDLQLVLPDYFPERSRLQREAEYFELPELVRRLGAPQQPGPGPPPSRRGVHKEGSLGDELLPLGYSEPEQQEGASAGAPSPTLELASRSPSGGAAGPLLTPSQSLDGSRRSGYITIGYRGSYTIGRDAQADAKFRRVARITVCGKTSLAKEVFGDTLNESRDPDRPPERYTSRYYLKFNFLEQAFDKLSESGFHMVACSSTGTCAFASSTDQSEDKIWTSYTEYVFCRE",
# LEG12_HUMAN
"MSQPSGGRAPGTRIYSWSCPTVMSPGEKLDPIPDSFILQPPVFHPVVPYVTTIFGGLHAGKMVMLQGVVPLDAHRFQVDFQCGCSLCPRPDIAFHFNPRFHTTKPHVICNTLHGGRWQREARWPHLALRRGSSFLILFLFGNEEVKVSVNGQHFLHFRYRLPLSHVDTLGIFGDILVEAVGFLNINPFVEGSREYPAGHPFLLMSPRLEVPCSHALPQGLSPGQVIIVRGLVLQEPKHFTVSLRDQAAHAPVTLRASFADRTLAWISRWGQKKLISAPFLFYPQRFFEVLLLFQEGGLKLALNGQGLGATSMNQQALEQLRELRISGSVQLYCVHS",
# RAB3C_HUMAN
"MRHEAPMQMASAQDARYGQKDSSDQNFDYMFKLLIIGNSSVGKTSFLFRYADDSFTSAFVSTVGIDFKVKTVFKNEKRIKLQIWDTAGQERYRTITTAYYRGAMGFILMYDITNEESFNAVQDWSTQIKTYSWDNAQVILVGNKCDMEDERVISTERGQHLGEQLGFEFFETSAKDNINVKQTFERLVDIICDKMSESLETDPAITAAKQNTRLKETPPPPQPNCAC",
# NTCP4_HUMAN
"MDGNDNVTLLFAPLLRDNYTLAPNASSLGPGTDLALAPASSAGPGPGLSLGPGPSFGFSPGPTPTPEPTTSGLAGGAASHGPSPFPRPWAPHALPFWDTPLNHGLNVFVGAALCITMLGLGCTVDVNHFGAHVRRPVGALLAALCQFGLLPLLAFLLALAFKLDEVAAVAVLLCGCCPGGNLSNLMSLLVDGDMNLSIIMTISSTLLALVLMPLCLWIYSWAWINTPIVQLLPLGTVTLTLCSTLIPIGLGVFIRYKYSRVADYIVKVSLWSLLVTLVVLFIMTGTMLGPELLASIPAAVYVIAIFMPLAGYASGYGLATLFHLPPNCKRTVCLETGSQNVQLCTAILKLAFPPQFIGSMYMFPLLYALFQSAEAGIFVLIYKMYGSEMLHKRDPLDEDEDTDISYKKLKEEEMADTSYGTVKAENIIMMETAQTSL",
# DTBP1_HUMAN
"MLETLRERLLSVQQDFTSGLKTLSDKSREAKVKSKPRTVPFLPKYSAGLELLSRYEDTWAALHRRAKDCASAGELVDSEVVMLSAHWEKKKTSLVELQEQLQQLPALIADLESMTANLTHLEASFEEVENNLLHLEDLCGQCELERCKHMQSQQLENYKKNKRKELETFKAELDAEHAQKVLEMEHTQQMKLKERQKFFEEAFQQDMEQYLSTGYLQIAERREPIGSMSSMEVNVDMLEQMDLMDISDQEALDVFLNSGGEENTVLSPALGPESSTCQNEITLQVPNPSELRAKPPSSSSTCTDSATRDISEGGESPVVQSDEEEVQVDTALATSHTDREATPDGGEDSDS",
# CYFP2_HUMAN
"MTTHVTLEDALSNVDLLEELPLPDQQPCIEPPPSSIMYQANFDTNFEDRNAFVTGIARYIEQATVHSSMNEMLEEGHEYAVMLYTWRSCSRAIPQVKCNEQPNRVEIYEKTVEVLEPEVTKLMKFMYFQRKAIERFCSEVKRLCHAERRKDFVSEAYLLTLGKFINMFAVLDELKNMKCSVKNDHSAYKRAAQFLRKMADPQSIQESQNLSMFLANHNRITQCLHQQLEVIPGYEELLADIVNICVDYYENKMYLTPSEKHMLLKVMGFGLYLMDGNVSNIYKLDAKKRINLSKIDKFFKQLQVVPLFGDMQIELARYIKTSAHYEENKSKWTCTQSSISPQYNICEQMVQIRDDHIRFISELARYSNSEVVTGSGLDSQKSDEEYRELFDLALRGLQLLSKWSAHVMEVYSWKLVHPTDKFCNKDCPGTAEEYERATRYNYTSEEKFAFVEVIAMIKGLQVLMGRMESVFNQAIRNTIYAALQDFAQVTLREPLRQAVRKKKNVLISVLQAIRKTICDWEGGREPPNDPCLRGEKDPKGGFDIKVPRRAVGPSSTQACQWSPRALFHPTGGTQGRRGCRSLLYMVRTMLESLIADKSGSKKTLRSSLDGPIVLAIEDFHKQSFFFTHLLNISEALQQCCDLSQLWFREFFLELTMGRRIQFPIEMSMPWILTDHILETKEPSMMEYVLYPLDLYNDSAYYALTKFKKQFLYDEIEAEVNLCFDQFVYKLADQIFAYYKAMAGSVLLDKRFRAECKNYGVIIPYPPSNRYETLLKQRHVQLLGRSIDLNRLITQRISAAMYKSLDQAISRFESEDLTSIVELEWLLEINRLTHRLLCKHMTLDSFDAMFREANHNVSAPYGRITLHVFWELNFDFLPNYCYNGSTNRFVRTAIPFTQEPQRDKPANVQPYYLYGSKPLNIAYSHIYSSYRNFVGPPHFKTICRLLGYQGIAVVMEELLKIVKSLLQGTILQYVKTLIEVMPKICRLPRHEYGSPGILEFFHHQLKDIIEYAELKTDVFQSLREVGNAILFCLLIEQALSQEEVCDLLHAAPFQNILPRVYIKEGERLEVRMKRLEAKYAPLHLVPLIERLGTPQQIAIAREGDLLTKERLCCGLSMFEVILTRIRSYLQDPIWRGPPPTNGVMHVDECVEFHRLWSAMQFVYCIPVGTNEFTAEQCFGDGLNWAGCSIIVLLGQQRRFDLFDFCYHLLKVQRQDGKDEIIKNVPLKKMADRIRKYQILNNEVFAILNKYMKSVETDSSTVEHVRCFQPPIHQSLATTC",
# CNRP1_HUMAN
"MGDLPGLVRLSIALRIQPNDGPVFYKVDGQRFGQNRTIKLLTGSSYKVEVKIKPSTLQVENISIGGVLVPLELKSKEPDGDRVVYTGTYDTEGVTPTKSGERQPIQITMPFTDIGTFETVWQVKFYNYHKRDHCQWGSPFSVIEYECKPNETRSLMWVNKESFL",
# LIGO1_HUMAN
"MQVSKRMLAGGVRSMPSPLLACWQPILLLVLGSVLSGSATGCPPRCECSAQDRAVLCHRKRFVAVPEGIPTETRLLDLGKNRIKTLNQDEFASFPHLEELELNENIVSAVEPGAFNNLFNLRTLGLRSNRLKLIPLGVFTGLSNLTKLDISENKIVILLDYMFQDLYNLKSLEVGDNDLVYISHRAFSGLNSLEQLTLEKCNLTSIPTEALSHLHGLIVLRLRHLNINAIRDYSFKRLYRLKVLEISHWPYLDTMTPNCLYGLNLTSLSITHCNLTAVPYLAVRHLVYLRFLNLSYNPISTIEGSMLHELLRLQEIQLVGGQLAVVEPYAFRGLNYLRVLNVSGNQLTTLEESVFHSVGNLETLILDSNPLACDCRLLWVFRRRWRLNFNRQQPTCATPEFVQGKEFKDFPDVLLPNYFTCRRARIRDRKAQQVFVDEGHTVQFVCRADGDPPPAILWLSPRKHLVSAKSNGRLTVFPDGTLEVRYAQVQDNGTYLCIAANAGGNDSMPAHLHVRSYSPDWPHQPNKTFAFISNQPGEGEANSTRATVPFPFDIKTLIIATTMGFISFLGVVLFCLVLLFLWSRGKGNTKHNIEIEYVPRKSDAGISSADAPRKFNMKMI",
# DYL2_HUMAN
"MSDRKAVIKNADMSEDMQQDAVDCATQAMEKYNIEKDIAAYIKKEFDKKYNPTWHCIVGRNFGSYVTHETKHFIYFYLGQVAILLFKSG",
# PLPP_HUMAN
"MARCERLRGAALRDVLGRAQGVLFDCDGVLWNGERAVPGAPELLERLARAGKAALFVSNNSRRARPELALRFARLGFGGLRAEQLFSSALCAARLLRQRLPGPPDAPGAVFVLGGEGLRAELRAAGLRLAGDPSAGDGAAPRVRAVLVGYDEHFSFAKLREACAHLRDPECLLVATDRDPWHPLSDGSRTPGTGSLAAAVETASGRQALVVGKPSPYMFECITENFSIDPARTLMVGDRLETDILFGHRCGMTTVLTLTGVSRLEEAQAYLAAGQHDLVPHYYVESIADLTEGLED",
# AB17A_HUMAN
"MNGLSLSELCCLFCCPPCPGRIAAKLAFLPPEATYSLVPEPEPGPGGAGAAPLGTLRASSGAPGRWKLHLTERADFQYSQRELDTIEVFPTKSARGNRVSCMYVRCVPGARYTVLFSHGNAVDLGQMSSFYIGLGSRLHCNIFSYDYSGYGASSGRPSERNLYADIDAAWQALRTRYGISPDSIILYGQSIGTVPTVDLASRYECAAVVLHSPLTSGMRVAFPDTKKTYCFDAFPNIEKVSKITSPVLIIHGTEDEVIDFSHGLALYERCPKAVEPLWVEGAGHNDIELYSQYLERLRRFISQELPSQRA",
# PGCB_HUMAN
"MAQLFLPLLAALVLAQAPAALADVLEGDSSEDRAFRVRIAGDAPLQGVLGGALTIPCHVHYLRPPPSRRAVLGSPRVKWTFLSRGREAEVLVARGVRVKVNEAYRFRVALPAYPASLTDVSLALSELRPNDSGIYRCEVQHGIDDSSDAVEVKVKGVVFLYREGSARYAFSFSGAQEACARIGAHIATPEQLYAAYLGGYEQCDAGWLSDQTVRYPIQTPREACYGDMDGFPGVRNYGVVDPDDLYDVYCYAEDLNGELFLGDPPEKLTLEEARAYCQERGAEIATTGQLYAAWDGGLDHCSPGWLADGSVRYPIVTPSQRCGGGLPGVKTLFLFPNQTGFPNKHSRFNVYCFRDSAQPSAIPEASNPASNPASDGLEAIVTVTETLEELQLPQEATESESRGAIYSIPIMEDGGGGSSTPEDPAEAPRTLLEFETQSMVPPTGFSEEEGKALEEEEKYEDEEEKEEEEEEEEVEDEALWAWPSELSSPGPEASLPTEPAAQEESLSQAPARAVLQPGASPLPDGESEASRPPRVHGPPTETLPTPRERNLASPSPSTLVEAREVGEATGGPELSGVPRGESEETGSSEGAPSLLPATRAPEGTRELEAPSEDNSGRTAPAGTSVQAQPVLPTDSASRGGVAVVPASGDCVPSPCHNGGTCLEEEEGVRCLCLPGYGGDLCDVGLRFCNPGWDAFQGACYKHFSTRRSWEEAETQCRMYGAHLASISTPEEQDFINNRYREYQWIGLNDRTIEGDFLWSDGVPLLYENWNPGQPDSYFLSGENCVVMVWHDQGQWSDVPCNYHLSYTCKMGLVSCGPPPELPLAQVFGRPRLRYEVDTVLRYRCREGLAQRNLPLIRCQENGRWEAPQISCVPRRPARALHPEEDPEGRQGRLLGRWKALLIPPSSPMPGP",
# PDLI5_HUMAN
"MSNYSVSLVGPAPWGFRLQGGKDFNMPLTISSLKDGGKAAQANVRIGDVVLSIDGINAQGMTHLEAQNKIKGCTGSLNMTLQRASAAPKPEPVPVQKGEPKEVVKPVPITSPAVSKVTSTNNMAYNKAPRPFGSVSSPKVTSIPSPSSAFTPAHATTSSHASPSPVAAVTPPLFAASGLHANANLSADQSPSALSAGKTAVNVPRQPTVTSVCSETSQELAEGQRRGSQGDSKQQNGPPRKHIVERYTEFYHVPTHSDASKKRLIEDTEDWRPRTGTTQSRSFRILAQITGTEHLKESEADNTKKANNSQEPSPQLASSVASTRSMPESLDSPTSGRPGVTSLTAAAAFKPVGSTGVIKSPSWQRPNQGVPSTGRISNSATYSGSVAPANSALGQTQPSDQDTLVQRAEHIPAGKRTPMCAHCNQVIRGPFLVALGKSWHPEEFNCAHCKNTMAYIGFVEEKGALYCELCYEKFFAPECGRCQRKILGEVISALKQTWHVSCFVCVACGKPIRNNVFHLEDGEPYCETDYYALFGTICHGCEFPIEAGDMFLEALGYTWHDTCFVCSVCCESLEGQTFFSKKDKPLCKKHAHSVNF",
# CREL1_HUMAN
"MAPWPPKGLVPAMLWGLSLFLNLPGPIWLQPSPPPQSSPPPQPHPCHTCRGLVDSFNKGLERTIRDNFGGGNTAWEEENLSKYKDSETRLVEVLEGVCSKSDFECHRLLELSEELVESWWFHKQQEAPDLFQWLCSDSLKLCCPAGTFGPSCLPCPGGTERPCGGYGQCEGEGTRGGSGHCDCQAGYGGEACGQCGLGYFEAERNASHLVCSACFGPCARCSGPEESNCLQCKKGWALHHLKCVDIDECGTEGANCGADQFCVNTEGSYECRDCAKACLGCMGAGPGRCKKCSPGYQQVGSKCLDVDECETEVCPGENKQCENTEGGYRCICAEGYKQMEGICVKEQIPESAGFFSEMTEDELVVLQQMFFGIIICALATLAAKGDLVFTAIFIGAVAAMTGYWLSERSDRVLEGFIKGR",
# SENP5_HUMAN
"MKKQRKILWRKGIHLAFSEKWNTGFGGFKKFYFHQHLCILKAKLGRPVTWNRQLRHFQGRKKALQIQKTWIKDEPLCAKTKFNVATQNVSTLSSKVKRKDAKHFISSSKTLLRLQAEKLLSSAKNSDHEYCREKNLLKAVTDFPSNSALGQANGHRPRTDPQPSDFPMKFNGESQSPGESGTIVVTLNNHKRKGFCYGCCQGPEHHRNGGPLIPKKFQLNQHRRIKLSPLMMYEKLSMIRFRYRILRSQHFRTKSKVCKLRKAQRSWVQKVTGDHQETRRENGEGGSCSPFPSPEPKDPSCRHQPYFPDMDSSAVVKGTNSHVPDCHTKGSSFLGKELSLDEAFPDQQNGSATNAWDQSSCSSPKWECTELIHDIPLPEHRSNTMFISETEREIMTLGQENQTSSVSDDRVKLSVSGADTSVSSVDGPVSQKAVQNENSYQMEEDGSLKQSILSSELLDHPYCKSPLEAPLVCSGLKLENQVGGGKNSQKASPVDDEQLSVCLSGFLDEVMKKYGSLVPLSEKEVLGRLKDVFNEDFSNRKPFINREITNYRARHQKCNFRIFYNKHMLDMDDLATLDGQNWLNDQVINMYGELIMDAVPDKVHFFNSFFHRQLVTKGYNGVKRWTKKVDLFKKSLLLIPIHLEVHWSLITVTLSNRIISFYDSQGIHFKFCVENIRKYLLTEAREKNRPEFLQGWQTAVTKCIPQQKNDSDCGVFVLQYCKCLALEQPFQFSQEDMPRVRKRIYKELCECRLMD",
# IGS21_HUMAN
"MRTAPSLRRCVCLLLAAILDLARGYLTVNIEPLPPVVAGDAVTLKCNFKTDGRMREIVWYRVTDGGTIKQKIFTFDAMFSTNYSHMENYRKREDLVYQSTVRLPEVRISDNGPYECHVGIYDRATREKVVLASGNIFLNVMAPPTSIEVVAADTPAPFSRYQAQNFTLVCIVSGGKPAPMVYFKRDGEPIDAVPLSEPPAASSGPLQDSRPFRSLLHRDLDDTKMQKSLSLLDAENRGGRPYTERPSRGLTPDPNILLQPTTENIPETVVSREFPRWVHSAEPTYFLRHSRTPSSDGTVEVRALLTWTLNPQIDNEALFSCEVKHPALSMPMQAEVTLVAPKGPKIVMTPSRARVGDTVRILVHGFQNEVFPEPMFTWTRVGSRLLDGSAEFDGKELVLERVPAELNGSMYRCTAQNPLGSTDTHTRLIVFENPNIPRGTEDSNGSIGPTGARLTLVLALTVILELT",
# FXL20_HUMAN
"MRRDVNGVTKSRFEMFSNSDEAVINKKLPKELLLRIFSFLDVVTLCRCAQVSRAWNVLALDGSNWQRIDLFDFQRDIEGRVVENISKRCGGFLRKLSLRGCLGVGDNALRTFAQNCRNIEVLNLNGCTKTTDATCTSLSKFCSKLRHLDLASCTSITNMSLKALSEGCPLLEQLNISWCDQVTKDGIQALVRGCGGLKALFLKGCTQLEDEALKYIGAHCPELVTLNLQTCLQITDEGLITICRGCHKLQSLCASGCSNITDAILNALGQNCPRLRILEVARCSQLTDVGFTTLARNCHELEKMDLEECVQITDSTLIQLSIHCPRLQVLSLSHCELITDDGIRHLGNGACAHDQLEVIELDNCPLITDASLEHLKSCHSLERIELYDCQQITRAGIKRLRTHLPNIKVHAYFAPVTPPPSVGGSRQRFCRCCIIL",
# VCIP1_HUMAN
"MSQPPPPPPPLPPPPPPPEAPQTPSSLASAAASGGLLKRRDRRILSGSCPDPKCQARLFFPASGSVSIECTECGQRHEQQQLLGVEEVTDPDVVLHNLLRNALLGVTGAPKKNTELVKVMGLSNYHCKLLSPILARYGMDKQTGRAKLLRDMNQGELFDCALLGDRAFLIEPEHVNTVGYGKDRSGSLLYLHDTLEDIKRANKSQECLIPVHVDGDGHCLVHAVSRALVGRELFWHALRENLKQHFQQHLARYQALFHDFIDAAEWEDIINECDPLFVPPEGVPLGLRNIHIFGLANVLHRPIILLDSLSGMRSSGDYSATFLPGLIPAEKCTGKDGHLNKPICIAWSSSGRNHYIPLVGIKGAALPKLPMNLLPKAWGVPQDLIKKYIKLEEDGGCVIGGDRSLQDKYLLRLVAAMEEVFMDKHGIHPSLVADVHQYFYRRTGVIGVQPEEVTAAAKKAVMDNRLHKCLLCGALSELHVPPEWLAPGGKLYNLAKSTHGQLRTDKNYSFPLNNLVCSYDSVKDVLVPDYGMSNLTACNWCHGTSVRKVRGDGSIVYLDGDRTNSRSTGGKCGCGFKHFWDGKEYDNLPEAFPITLEWGGRVVRETVYWFQYESDSSLNSNVYDVAMKLVTKHFPGEFGSEILVQKVVHTILHQTAKKNPDDYTPVNIDGAHAQRVGDVQGQESESQLPTKIILTGQKTKTLHKEELNMSKTERTIQQNITEQASVMQKRKTEKLKQEQKGQPRTVSPSTIRDGPSSAPATPTKAPYSPTTSKEKKIRITTNDGRQSMVTLKSSTTFFELQESIAREFNIPPYLQCIRYGFPPKELMPPQAGMEKEPVPLQHGDRITIEILKSKAEGGQSAAAHSAHTVKQEDIAVTGKLSSKELQEQAEKEMYSLCLLATLMGEDVWSYAKGLPHMFQQGGVFYSIMKKTMGMADGKHCTFPHLPGKTFVYNASEDRLELCVDAAGHFPIGPDVEDLVKEAVSQVRAEATTRSRESSPSHGLLKLGSGGVVKKKSEQLHNVTAFQGKGHSLGTASGNPHLDPRARETSVVRKHNTGTDFSNSSTKTEPSVFTASSSNSELIRIAPGVVTMRDGRQLDPDLVEAQRKKLQEMVSSIQASMDRHLRDQSTEQSPSDLPQRKTEVVSSSAKSGSLQTGLPESFPLTGGTENLNTETTDGCVADALGAAFATRSKAQRGNSVEELEEMDSQDAEMTNTTEPMDHS",
# EXOC2_HUMAN
"MSRSRQPPLVTGISPNEGIPWTKVTIRGENLGTGPTDLIGLTICGHNCLLTAEWMSASKIVCRVGQAKNDKGDIIVTTKSGGRGTSTVSFKLLKPEKIGILDQSAVWVDEMNYYDMRTDRNKGIPPLSLRPANPLGIEIEKSKFSQKDLEMLFHGMSADFTSENFSAAWYLIENHSNTSFEQLKMAVTNLKRQANKKSEGSLAYVKGGLSTFFEAQDALSAIHQKLEADGTEKVEGSMTQKLENVLNRASNTADTLFQEVLGRKDKADSTRNALNVLQRFKFLFNLPLNIERNIQKGDYDVVINDYEKAKSLFGKTEVQVFKKYYAEVETRIEALRELLLDKLLETPSTLHDQKRYIRYLSDLHASGDPAWQCIGAQHKWILQLMHSCKEGYVKDLKGNPGLHSPMLDLDNDTRPSVLGHLSQTASLKRGSSFQSGRDDTWRYKTPHRVAFVEKLTKLVLSQLPNFWKLWISYVNGSLFSETAEKSGQIERSKNVRQRQNDFKKMIQEVMHSLVKLTRGALLPLSIRDGEAKQYGGWEVKCELSGQWLAHAIQTVRLTHESLTALEIPNDLLQTIQDLILDLRVRCVMATLQHTAEEIKRLAEKEDWIVDNEGLTSLPCQFEQCIVCSLQSLKGVLECKPGEASVFQQPKTQEEVCQLSINIMQVFIYCLEQLSTKPDADIDTTHLSVDVSSPDLFGSIHEDFSLTSEQRLLIVLSNCCYLERHTFLNIAEHFEKHNFQGIEKITQVSMASLKELDQRLFENYIELKADPIVGSLEPGIYAGYFDWKDCLPPTGVRNYLKEALVNIIAVHAEVFTISKELVPRVLSKVIEAVSEELSRLMQCVSSFSKNGALQARLEICALRDTVAVYLTPESKSSFKQALEALPQLSSGADKKLLEELLNKFKSSMHLQLTCFQAASSTMMKT",
# SNX27_HUMAN
"MADEDGEGIHPSAPHRNGGGGGGGGSGLHCAGNGGGGGGGPRVVRIVKSESGYGFNVRGQVSEGGQLRSINGELYAPLQHVSAVLPGGAADRAGVRKGDRILEVNHVNVEGATHKQVVDLIRAGEKELILTVLSVPPHEADNLDPSDDSLGQSFYDYTEKQAVPISVPRYKHVEQNGEKFVVYNVYMAGRQLCSKRYREFAILHQNLKREFANFTFPRLPGKWPFSLSEQQLDARRRGLEEYLEKVCSIRVIGESDIMQEFLSESDENYNGVSDVELRVALPDGTTVTVRVKKNSTTDQVYQAIAAKVGMDSTTVNYFALFEVISHSFVRKLAPNEFPHKLYIQNYTSAVPGTCLTIRKWLFTTEEEILLNDNDLAVTYFFHQAVDDVKKGYIKAEEKSYQLQKLYEQRKMVMYLNMLRTCEGYNEIIFPHCACDSRRKGHVITAISITHFKLHACTEEGQLENQVIAFEWDEMQRWDTDEEGMAFCFEYARGEKKPRWVKIFTPYFNYMHECFERVFCELKWRKENIFQMARSQQRDVAT",
# TRI47_HUMAN
"MDGSGPFSCPICLEPLREPVTLPCGHNFCLACLGALWPHRGASGAGGPGGAARCPLCQEPFPDGLQLRKNHTLSELLQLRQGSGPGSGPGPAPALAPEPSAPSALPSVPEPSAPCAPEPWPAGEEPVRCDACPEGAALPAALSCLSCLASFCPAHLGPHERSPALRGHRLVPPLRRLEESLCPRHLRPLERYCRAERVCLCEACAAQEHRGHELVPLEQERALQEAEQSKVLSAVEDRMDELGAGIAQSRRTVALIKSAAVAERERVSRLFADAAAALQGFQTQVLGFIEEGEAAMLGRSQGDLRRQEEQRSRLSRARQNLSQVPEADSVSFLQELLALRLALEDGCGPGPGPPRELSFTKSSQAVRAVRDMLAVACVNQWEQLRGPGGNEDGPQKLDSEADAEPQDLESTNLLESEAPRDYFLKFAYIVDLDSDTADKFLQLFGTKGVKRVLCPINYPLSPTRFTHCEQVLGEGALDRGTYYWEVEIIEGWVSMGVMAEDFSPQEPYDRGRLGRNAHSCCLQWNGRSFSVWFHGLEAPLPHPFSPTVGVCLEYADRALAFYAVRDGKMSLLRRLKASRPRRGGIPASPIDPFQSRLDSHFAGLFTHRLKPAFFLESVDAHLQIGPLKKSCISVLKRR",
# TAFA4_HUMAN
"MRSPRMRVCAKSVLLSHWLFLAYVLMVCCKLMSASSQHLRGHAGHHQIKQGTCEVVAVHRCCNKNRIEERSQTVKCSCFPGQVAGTTRAQPSCVEASIVIQKWWCHMNPCLEGEDCKVLPDYSGWSCSSGNKVKTTKVTR",
# CI072_HUMAN
"MSTLCPPPSPAVAKTEIALSGKSPLLAATFAYWDNILGPRVRHIWAPKTEQVLLSDGEITFLANHTLNGEILRNAESGAIDVKFFVLSEKGVIIVSLIFDGNWNGDRSTYGLSIILPQTELSFYLPLHRVCVDRLTHIIRKGRIWMHKERQENVQKIILEGTERMEDQGQSIIPMLTGEVIPVMELLSSMKSHSVPEEIDIADTVLNDDDIGDSCHEGFLLNAISSHLQTCGCSVVVGSSAEKVNKIVRTLCLFLTPAERKCSRLCEAESSFKYESGLFVQGLLKDSTGSFVLPFRQVMYAPYPTTHIDVDVNTVKQMPPCHEHIYNQRRYMRSELTAFWRATSEEDMAQDTIIYTDESFTPDLNIFQDVLHRDTLVKAFLDQVFQLKPGLSLRSTFLAQFLLVLHRKALTLIKYIEDDTQKGKKPFKSLRNLKIDLDLTAEGDLNIIMALAEKIKPGLHSFIFGRPFYTSVQERDVLMTF",
# PRIC1_HUMAN
"MPLEMEPKMSKLAFGCQRSSTSDDDSGCALEEYAWVPPGLRPEQIQLYFACLPEEKVPYVNSPGEKHRIKQLLYQLPPHDNEVRYCQSLSEEEKKELQVFSAQRKKEALGRGTIKLLSRAVMHAVCEQCGLKINGGEVAVFASRAGPGVCWHPSCFVCFTCNELLVDLIYFYQDGKIHCGRHHAELLKPRCSACDEIIFADECTEAEGRHWHMKHFCCLECETVLGGQRYIMKDGRPFCCGCFESLYAEYCETCGEHIGVDHAQMTYDGQHWHATEACFSCAQCKASLLGCPFLPKQGQIYCSKTCSLGEDVHASDSSDSAFQSARSRDSRRSVRMGKSSRSADQCRQSLLLSPALNYKFPGLSGNADDTLSRKLDDLSLSRQGTSFASEEFWKGRVEQETPEDPEEWADHEDYMTQLLLKFGDKSLFQPQPNEMDIRASEHWISDNMVKSKTELKQNNQSLASKKYQSDMYWAQSQDGLGDSAYGSHPGPASSRRLQELELDHGASGYNHDETQWYEDSLECLSDLKPEQSVRDSMDSLALSNITGASVDGENKPRPSLYSLQNFEEMETEDCEKMSNMGTLNSSMLHRSAESLKSLSSELCPEKILPEEKPVHLPVLRRSKSQSRPQQVKFSDDVIDNGNYDIEIRQPPMSERTRRRVYNFEERGSRSHHHRRRRSRKSRSDNALNLVTERKYSPKDRLRLYTPDNYEKFIQNKSAREIQAYIQNADLYGQYAHATSDYGLQNPGMNRFLGLYGEDDDSWCSSSSSSSDSEEEGYFLGQPIPQPRPQRFAYYTDDLSSPPSALPTPQFGQRTTKSKKKKGHKGKNCIIS",
# DOCK7_HUMAN
"MAERRAFAQKISRTVAAEVRKQISGQYSGSPQLLKNLNIVGNISHHTTVPLTEAVDPVDLEDYLITHPLAVDSGPLRDLIEFPPDDIEVVYSPRDCRTLVSAVPEESEMDPHVRDCIRSYTEDWAIVIRKYHKLGTGFNPNTLDKQKERQKGLPKQVFESDEAPDGNSYQDDQDDLKRRSMSIDDTPRGSWACSIFDLKNSLPDALLPNLLDRTPNEEIDRQNDDQRKSNRHKELFALHPSPDEEEPIERLSVPDIPKEHFGQRLLVKCLSLKFEIEIEPIFASLALYDVKEKKKISENFYFDLNSEQMKGLLRPHVPPAAITTLARSAIFSITYPSQDVFLVIKLEKVLQQGDIGECAEPYMIFKEADATKNKEKLEKLKSQADQFCQRLGKYRMPFAWTAIHLMNIVSSAGSLERDSTEVEISTGERKGSWSERRNSSIVGRRSLERTTSGDDACNLTSFRPATLTVTNFFKQEGDRLSDEDLYKFLADMRRPSSVLRRLRPITAQLKIDISPAPENPHYCLTPELLQVKLYPDSRVRPTREILEFPARDVYVPNTTYRNLLYIYPQSLNFANRQGSARNITVKVQFMYGEDPSNAMPVIFGKSSCSEFSKEAYTAVVYHNRSPDFHEEIKVKLPATLTDHHHLLFTFYHVSCQQKQNTPLETPVGYTWIPMLQNGRLKTGQFCLPVSLEKPPQAYSVLSPEVPLPGMKWVDNHKGVFNVEVVAVSSIHTQDPYLDKFFALVNALDEHLFPVRIGDMRIMENNLENELKSSISALNSSQLEPVVRFLHLLLDKLILLVIRPPVIAGQIVNLGQASFEAMASIINRLHKNLEGNHDQHGRNSLLASYIHYVFRLPNTYPNSSSPGPGGLGGSVHYATMARSAVRPASLNLNRSRSLSNSNPDISGTPTSPDDEVRSIIGSKGLDRSNSWVNTGGPKAAPWGSNPSPSAESTQAMDRSCNRMSSHTETSSFLQTLTGRLPTKKLFHEELALQWVVCSGSVRESALQQAWFFFELMVKSMVHHLYFNDKLEAPRKSRFPERFMDDIAALVSTIASDIVSRFQKDTEMVERLNTSLAFFLNDLLSVMDRGFVFSLIKSCYKQVSSKLYSLPNPSVLVSLRLDFLRIICSHEHYVTLNLPCSLLTPPASPSPSVSSATSQSSGFSTNVQDQKIANMFELSVPFRQQHYLAGLVLTELAVILDPDAEGLFGLHKKVINMVHNLLSSHDSDPRYSDPQIKARVAMLYLPLIGIIMETVPQLYDFTETHNQRGRPICIATDDYESESGSMISQTVAMAIAGTSVPQLTRPGSFLLTSTSGRQHTTFSAESSRSLLICLLWVLKNADETVLQKWFTDLSVLQLNRLLDLLYLCVSCFEYKGKKVFERMNSLTFKKSKDMRAKLEEAILGSIGARQEMVRRSRGQLGTYTIASPPERSPSGSAFGSQENLRWRKDMTHWRQNTEKLDKSRAEIEHEALIDGNLATEANLIILDTLEIVVQTVSVTESKESILGGVLKVLLHSMACNQSAVYLQHCFATQRALVSKFPELLFEEETEQCADLCLRLLRHCSSSIGTIRSHASASLYLLMRQNFEIGNNFARVKMQVTMSLSSLVGTSQNFNEEFLRRSLKTILTYAEEDLELRETTFPDQVQDLVFNLHMILSDTVKMKEHQEDPEMLIDLMYRIAKGYQTSPDLRLTWLQNMAGKHSERSNHAEAAQCLVHSAALVAEYLSMLEDRKYLPVGCVTFQNISSNVLEESAVSDDVVSPDEEGICSGKYFTESGLVGLLEQAAASFSMAGMYEAVNEVYKVLIPIHEANRDAKKLSTIHGKLQEAFSKIVHQSTGWERMFGTYFRVGFYGTKFGDLDEQEFVYKEPAITKLAEISHRLEGFYGERFGEDVVEVIKDSNPVDKCKLDPNKAYIQITYVEPYFDTYEMKDRITYFDKNYNLRRFMYCTPFTLDGRAHGELHEQFKRKTILTTSHAFPYIKTRVNVTHKEEIILTPIEVAIEDMQKKTQELAFATHQDPADPKMLQMVLQGSVGTTVNQGPLEVAQVFLSEIPSDPKLFRHHNKLRLCFKDFTKRCEDALRKNKSLIGPDQKEYQRELERNYHRLKEALQPLINRKIPQLYKAVLPVTCHRDSFSRMSLRKMDL",
# LRFN5_HUMAN
"MEKILFYLFLIGIAVKAQICPKRCVCQILSPNLATLCAKKGLLFVPPNIDRRTVELRLADNFVTNIKRKDFANMTSLVDLTLSRNTISFITPHAFADLRNLRALHLNSNRLTKITNDMFSGLSNLHHLILNNNQLTLISSTAFDDVFALEELDLSYNNLETIPWDAVEKMVSLHTLSLDHNMIDNIPKGTFSHLHKMTRLDVTSNKLQKLPPDPLFQRAQVLATSGIISPSTFALSFGGNPLHCNCELLWLRRLSREDDLETCASPPLLTGRYFWSIPEEEFLCEPPLITRHTHEMRVLEGQRATLRCKARGDPEPAIHWISPEGKLISNATRSLVYDNGTLDILITTVKDTGAFTCIASNPAGEATQIVDLHIIKLPHLLNSTNHIHEPDPGSSDISTSTKSGSNTSSSNGDTKLSQDKIVVAEATSSTALLKFNFQRNIPGIRMFQIQYNGTYDDTLVYRMIPPTSKTFLVNNLAAGTMYDLCVLAIYDDGITSLTATRVVGCIQFTTEQDYVRCHFMQSQFLGGTMIIIIGGIIVASVLVFIIILMIRYKVCNNNGQHKVTKVSNVYSQTNGAQIQGCSVTLPQSVSKQAVGHEENAQCCKATSDNVIQSSETCSSQDSSTTTSALPPSWTSSTSVSQKQKRKTGTKPSTEPQNEAVTNVESQNTNRNNSTALQLASRPPDSVTEGPTSKRAHIKPNALLTNVDQIVQETQRLELI",
# LRRC7_HUMAN
"MTTKRKIIGRLVPCRCFRGEEEIISVLDYSHCSLQQVPKEVFNFERTLEELYLDANQIEELPKQLFNCQALRKLSIPDNDLSNLPTTIASLVNLKELDISKNGVQEFPENIKCCKCLTIIEASVNPISKLPDGFTQLLNLTQLYLNDAFLEFLPANFGRLVKLRILELRENHLKTLPKSMHKLAQLERLDLGNNEFGELPEVLDQIQNLRELWMDNNALQVLPGSIGKLKMLVYLDMSKNRIETVDMDISGCEALEDLLLSSNMLQQLPDSIGLLKKLTTLKVDDNQLTMLPNTIGNLSLLEEFDCSCNELESLPSTIGYLHSLRTLAVDENFLPELPREIGSCKNVTVMSLRSNKLEFLPEEIGQMQKLRVLNLSDNRLKNLPFSFTKLKELAALWLSDNQSKALIPLQTEAHPETKQRVLTNYMFPQQPRGDEDFQSDSDSFNPTLWEEQRQQRMTVAFEFEDKKEDDENAGKVKDLSCQAPWERGQRGITLQPARLSGDCCTPWARCDQQIQDMPVPQNDPQLAWGCISGLQQERSMCTPLPVAAQSTTLPSLSGRQVEINLKRYPTPYPEDLKNMVKSVQNLVGKPSHGVRVENSNPTANTEQTVKEKYEHKWPVAPKEITVEDSFVHPANEMRIGELHPSLAETPLYPPKLVLLGKDKKESTDESEVDKTHCLNNSVSSGTYSDYSPSQASSGSSNTRVKVGSLQTTAKDAVHNSLWGNRIAPSFPQPLDSKPLLSQREAVPPGNIPQRPDRLPMSDTFTDNWTDGSHYDNTGFVAEETTAENANSNPLLSSKSRSTSSHGRRPLIRQDRIVGVPLELEQSTHRHTPETEVPPSNPWQNWTRTPSPFEDRTAFPSKLETTPTTSPLPERKEHIKESTEIPSPFSPGVPWEYHDSNPNRSLSNVFSQIHCRPESSKGVISISKSTERLSPLMKDIKSNKFKKSQSIDEIDIGTYKVYNIPLENYASGSDHLGSHERPDKMLGPEHGMSSMSRSQSVPMLDDEMLTYGSSKGPQQQKASMTKKVYQFDQSFNPQGSVEVKAEKRIPPPFQHNPEYVQQASKNIAKDLISPRAYRGYPPMEQMFSFSQPSVNEDAVVNAQFASQGARAGFLRRADSLVSATEMAMFRRVNEPHELPPTDRYGRPPYRGGLDRQSSVTVTESQFLKRNGRYEDEHPSYQEVKAQAGSFPVKNLTQRRPLSARSYSTESYGASQTRPVSARPTMAALLEKIPSDYNLGNYGDKPSDNSDLKTRPTPVKGEESCGKMPADWRQQLLRHIEARRLDRNAAYKHNTVNLGMLPYGGISAMHAGRSMTLNLQTKSKFDHQELPLQKTPSQQSNILDNGQEDVSPSGQWNPYPLGRRDVPPDTITKKAGSHIQTLMGSQSLQHRSREQQPYEGNINKVTIQQFQSPLPIQIPSSQATRGPQPGRCLIQTKGQRSMDGYPEQFCVRIEKNPGLGFSISGGISGQGNPFKPSDKGIFVTRVQPDGPASNLLQPGDKILQANGHSFVHMEHEKAVLLLKSFQNTVDLVIQRELTV",
# AGAP3_HUMAN
"MNFQAGGGQSPQQQQQLAGGPPQQFALSNSAAIRAEIQRFESVHPNIYAIYDLIERIEDLALQNQIREHVISIEDSFVNSQEWTLSRSVPELKVGIVGNLSSGKSALVHRYLTGTYVQEESPEGGRFKKEIVVDGQSYLLLIRDEGGPPELQFAAWVDAVVFVFSLEDEISFQTVYNYFLRLCSFRNASEVPMVLVGTQDAISAANPRVIDDSRARKLSTDLKRCTYYETCATYGLNVERVFQDVAQKVVALRKKQQLAIGPCKSLPNSPSHSAVSAASIPAVHINQATNGGGSAFSDYSSSVPSTPSISQRELRIETIAASSTPTPIRKQSKRRSNIFTSRKGADLDREKKAAECKVDSIGSGRAIPIKQGILLKRSGKSLNKEWKKKYVTLCDNGLLTYHPSLHDYMQNIHGKEIDLLRTTVKVPGKRLPRATPATAPGTSPRANGLSVERSNTQLGGGTGAPHSASSASLHSERPLSSSAWAGPRPEGLHQRSCSVSSADQWSEATTSLPPGMQHPASGPAEVLSSSPKLDPPPSPHSNRKKHRRKKSTGTPRPDGPSSATEEAEESFEFVVVSLTGQTWHFEASTAEERELWVQSVQAQILASLQGCRSAKDKTRLGNQNAALAVQAVRTVRGNSFCIDCDAPNPDWASLNLGALMCIECSGIHRHLGAHLSRVRSLDLDDWPPELLAVMTAMGNALANSVWEGALGGYSKPGPDACREEKERWIRAKYEQKLFLAPLPSSDVPLGQQLLRAVVEDDLRLLVMLLAHGSKEEVNETYGDGDGRTALHLSSAMANVVFTQLLIWYGVDVRSRDARGLTPLAYARRAGSQECADILIQHGCPGEGCGLAPTPNREPANGTNPSAELHRSPSLL",
# INP4A_HUMAN
"MTAREHSPRHGARARAMQRASTIDVAADMLGLSLAGNIQDPDEPILEFSLACSELHTPSLDRKPNSFVAVSVTTPPQAFWTKHAQTEIIEGTNNPIFLSSIAFFQDSLINQMTQVKLSVYDVKDRSQGTMYLLGSGTFIVKDLLQDRHHRLHLTLRSAESDRVGNITVIGWQMEEKSDQRPPVTRSVDTVNGRMVLPVDESLTEALGIRSKYASLRKDTLLKSVFGGAICRMYRFPTTDGNHLRILEQMAESVLSLHVPRQFVKLLLEEDAARVCELEELGELSPCWESLRRQIVTQYQTIILTYQENLTDLHQYRGPSFKASSLKADKKLEFVPTNLHIQRMRVQDDGGSDQNYDIVTIGAPAAHCQGFKSGGLRKKLHKFEETKKHFEECCTSSGCQSIIYIPQDVVRAKEIIAQINTLKTQVSYYAERLSRAAKDRSATGLERTLAILADKTRQLVTVCDCKLLANSIHGLNAARPDYIASKASPTSTEEEQVMLRNDQDTLMARWTGRNSRSSLQVDWHEEEWEKVWLNVDKSLECIIQRVDKLLQKERLHGEGCEDVFPCAGSCTSKKGNPDSHAYWIRPEDPFCDVPSSPCPSTMPSTACHPHLTTHCSPPPEESSPGEWSEALYPLLTTLTDCVAMMSDKAKKAMVFLLMQDSAPTIATYLSLQYRRDVVFCQTLTALICGFIIKLRNCLHDDGFLRQLYTIGLLAQFESLLSTYGEELAMLEDMSLGIMDLRNVTFKVTQATSSASADMLPVITGNRDGFNVRVPLPGPLFDALPREIQSGMLLRVQPVLFNVGINEQQTLAERFGDTSLQEVINVESLVRLNSYFEQFKEVLPEDCLPRSRSQTCLPELLRFLGQNVHARKNKNVDILWQAAEICRRLNGVRFTSCKSAKDRTAMSVTLEQCLILQHEHGMAPQVFTQALECMRSEGCRRENTMKNVGSRKYAFNSLQLKAFPKHYRPPEGTYGKVET",
# KCNC2_HUMAN
"MGKIENNERVILNVGGTRHETYRSTLKTLPGTRLALLASSEPPGDCLTTAGDKLQPSPPPLSPPPRAPPLSPGPGGCFEGGAGNCSSRGGRASDHPGGGREFFFDRHPGVFAYVLNYYRTGKLHCPADVCGPLFEEELAFWGIDETDVEPCCWMTYRQHRDAEEALDIFETPDLIGGDPGDDEDLAAKRLGIEDAAGLGGPDGKSGRWRRLQPRMWALFEDPYSSRAARFIAFASLFFILVSITTFCLETHEAFNIVKNKTEPVINGTSVVLQYEIETDPALTYVEGVCVVWFTFEFLVRIVFSPNKLEFIKNLLNIIDFVAILPFYLEVGLSGLSSKAAKDVLGFLRVVRFVRILRIFKLTRHFVGLRVLGHTLRASTNEFLLLIIFLALGVLIFATMIYYAERVGAQPNDPSASEHTQFKNIPIGFWWAVVTMTTLGYGDMYPQTWSGMLVGALCALAGVLTIAMPVPVIVNNFGMYYSLAMAKQKLPRKRKKHIPPAPQASSPTFCKTELNMACNSTQSDTCLGKDNRLLEHNRSVLSGDDSTGSEPPLSPPERLPIRRSSTRDKNRRGETCFLLTTGDYTCASDGGIRKGYEKSRSLNNIAGLAGNALRLSPVTSPYNSPCPLRRSRSPIPSIL",
# NED4L_HUMAN
"MATGLGEPVYGLSEDEGESRILRVKVVSGIDLAKKDIFGASDPYVKLSLYVADENRELALVQTKTIKKTLNPKWNEEFYFRVNPSNHRLLFEVFDENRLTRDDFLGQVDVPLSHLPTEDPTMERPYTFKDFLLRPRSHKSRVKGFLRLKMAYMPKNGGQDEENSDQRDDMEHGWEVVDSNDSASQHQEELPPPPLPPGWEEKVDNLGRTYYVNHNNRTTQWHRPSLMDVSSESDNNIRQINQEAAHRRFRSRRHISEDLEPEPSEGGDVPEPWETISEEVNIAGDSLGLALPPPPASPGSRTSPQELSEELSRRLQITPDSNGEQFSSLIQREPSSRLRSCSVTDAVAEQGHLPPPSAPAGRARSSTVTGGEEPTPSVAYVHTTPGLPSGWEERKDAKGRTYYVNHNNRTTTWTRPIMQLAEDGASGSATNSNNHLIEPQIRRPRSLSSPTVTLSAPLEGAKDSPVRRAVKDTLSNPQSPQPSPYNSPKPQHKVTQSFLPPGWEMRIAPNGRPFFIDHNTKTTTWEDPRLKFPVHMRSKTSLNPNDLGPLPPGWEERIHLDGRTFYIDHNSKITQWEDPRLQNPAITGPAVPYSREFKQKYDYFRKKLKKPADIPNRFEMKLHRNNIFEESYRRIMSVKRPDVLKARLWIEFESEKGLDYGGVAREWFFLLSKEMFNPYYGLFEYSATDNYTLQINPNSGLCNEDHLSYFTFIGRVAGLAVFHGKLLDGFFIRPFYKMMLGKQITLNDMESVDSEYYNSLKWILENDPTELDLMFCIDEENFGQTYQVDLKPNGSEIMVTNENKREYIDLVIQWRFVNRVQKQMNAFLEGFTELLPIDLIKIFDENELELLMCGLGDVDVNDWRQHSIYKNGYCPNHPVIQWFWKAVLLMDAEKRIRLLQFVTGTSRVPMNGFAELYGSNGPQLFTIEQWGSPEKLPRAHTCFNRLDLPPYETFEDLREKLLMAVENAQGFEGVD",
# QKI_HUMAN
"MVGEMETKEKPKPTPDYLMQLMNDKKLMSSLPNFCGIFNHLERLLDEEISRVRKDMYNDTLNGSTEKRSAELPDAVGPIVQLQEKLYVPVKEYPDFNFVGRILGPRGLTAKQLEAETGCKIMVRGKGSMRDKKKEEQNRGKPNWEHLNEDLHVLITVEDAQNRAEIKLKRAVEEVKKLLVPAAEGEDSLKKMQLMELAILNGTYRDANIKSPALAFSLAATAQAAPRIITGPAPVLPPAALRTPTPAGPTIMPLIRQIQTAVMPNGTPHPTAAIVPPGPEAGLIYTPYEYPYTLAPATSILEYPIEPSGVLGAVATKVRRHDMRVHPYQRIVTADRAATGN",
# SYGP1_HUMAN
"MSRSRASIHRGSIPAMSYAPFRDVRGPSMHRTQYVHSPYDRPGWNPRFCIISGNQLLMLDEDEIHPLLIRDRRSESSRNKLLRRTVSVPVEGRPHGEHEYHLGRSRRKSVPGGKQYSMEGAPAAPFRPSQGFLSRRLKSSIKRTKSQPKLDRTSSFRQILPRFRSADHDRARLMQSFKESHSHESLLSPSSAAEALELNLDEDSIIKPVHSSILGQEFCFEVTTSSGTKCFACRSAAERDKWIENLQRAVKPNKDNSRRVDNVLKLWIIEARELPPKKRYYCELCLDDMLYARTTSKPRSASGDTVFWGEHFEFNNLPAVRALRLHLYRDSDKKRKKDKAGYVGLVTVPVATLAGRHFTEQWYPVTLPTGSGGSGGMGSGGGGGSGGGSGGKGKGGCPAVRLKARYQTMSILPMELYKEFAEYVTNHYRMLCAVLEPALNVKGKEEVASALVHILQSTGKAKDFLSDMAMSEVDRFMEREHLIFRENTLATKAIEEYMRLIGQKYLKDAIGEFIRALYESEENCEVDPIKCTASSLAEHQANLRMCCELALCKVVNSHCVFPRELKEVFASWRLRCAERGREDIADRLISASLFLRFLCPAIMSPSLFGLMQEYPDEQTSRTLTLIAKVIQNLANFSKFTSKEDFLGFMNEFLELEWGSMQQFLYEISNLDTLTNSSSFEGYIDLGRELSTLHALLWEVLPQLSKEALLKLGPLPRLLNDISTALRNPNIQRQPSRQSERPRPQPVVLRGPSAEMQGYMMRDLNSSIDLQSFMARGLNSSMDMARLPSPTKEKPPPPPPGGGKDLFYVSRPPLARSSPAYCTSSSDITEPEQKMLSVNKSVSMLDLQGDGPGGRLNSSSVSNLAAVGDLLHSSQASLTAALGLRPAPAGRLSQGSGSSITAAGMRLSQMGVTTDGVPAQQLRIPLSFQNPLFHMAADGPGPPGGHGGGGGHGPPSSHHHHHHHHHHRGGEPPGDTFAPFHGYSKSEDLSSGVPKPPAASILHSHSYSDEFGPSGTDFTRRQLSLQDNLQHMLSPPQITIGPQRPAPSGPGGGSGGGSGGGGGGQPPPLQRGKSQQLTVSAAQKPRPSSGNLLQSPEPSYGPARPRQQSLSKEGSIGGSGGSGGGGGGGLKPSITKQHSQTPSTLNPTMPASERTVAWVSNMPHLSADIESAHIEREEYKLKEYSKSMDESRLDRVKEYEEEIHSLKERLHMSNRKLEEYERRLLSQEEQTSKILMQYQARLEQSEKRLRQQQAEKDSQIKSIIGRLMLVEEELRRDHPAMAEPLPEPKKRLLDAQERQLPPLGPTNPRVTLAPPWNGLAPPAPPPPPRLQITENGEFRNTADH",
# SLIK1_HUMAN
"MLLWILLLETSLCFAAGNVTGDVCKEKICSCNEIEGDLHVDCEKKGFTSLQRFTAPTSQFYHLFLHGNSLTRLFPNEFANFYNAVSLHMENNGLHEIVPGAFLGLQLVKRLHINNNKIKSFRKQTFLGLDDLEYLQADFNLLRDIDPGAFQDLNKLEVLILNDNLISTLPANVFQYVPITHLDLRGNRLKTLPYEEVLEQIPGIAEILLEDNPWDCTCDLLSLKEWLENIPKNALIGRVVCEAPTRLQGKDLNETTEQDLCPLKNRVDSSLPAPPAQEETFAPGPLPTPFKTNGQEDHATPGSAPNGGTKIPGNWQIKIRPTAAIATGSSRNKPLANSLPCPGGCSCDHIPGSGLKMNCNNRNVSSLADLKPKLSNVQELFLRDNKIHSIRKSHFVDYKNLILLDLGNNNIATVENNTFKNLLDLRWLYMDSNYLDTLSREKFAGLQNLEYLNVEYNAIQLILPGTFNAMPKLRILILNNNLLRSLPVDVFAGVSLSKLSLHNNYFMYLPVAGVLDQLTSIIQIDLHGNPWECSCTIVPFKQWAERLGSEVLMSDLKCETPVNFFRKDFMLLSNDEICPQLYARISPTLTSHSKNSTGLAETGTHSNSYLDTSRVSISVLVPGLLLVFVTSAFTVVGMLVFILRNRKRSKRRDANSSASEINSLQTVCDSSYWHNGPYNADGAHRVYDCGSHSLSD",
# FMNL2_HUMAN
"MGNAGSMDSQQTDFRAHNVPLKLPMPEPGELEERFAIVLNAMNLPPDKARLLRQYDNEKKWELICDQERFQVKNPPHTYIQKLKGYLDPAVTRKKFRRRVQESTQVLRELEISLRTNHIGWVREFLNEENKGLDVLVEYLSFAQYAVTFDFESVESTVESSVDKSKPWSRSIEDLHRGSNLPSPVGNSVSRSGRHSALRYNTLPSRRTLKNSRLVSKKDDVHVCIMCLRAIMNYQYGFNMVMSHPHAVNEIALSLNNKNPRTKALVLELLAAVCLVRGGHEIILSAFDNFKEVCGEKQRFEKLMEHFRNEDNNIDFMVASMQFINIVVHSVEDMNFRVHLQYEFTKLGLDEYLDKLKHTESDKLQVQIQAYLDNVFDVGALLEDAETKNAALERVEELEENISHLSEKLQDTENEAMSKIVELEKQLMQRNKELDVVREIYKDANTQVHTLRKMVKEKEEAIQRQSTLEKKIHELEKQGTIKIQKKGDGDIAILPVVASGTLSMGSEVVAGNSVGPTMGAASSGPLPPPPPPLPPSSDTPETVQNGPVTPPMPPPPPPPPPPPPPPPPPPPPPLPGPAAETVPAPPLAPPLPSAPPLPGTSSPTVVFNSGLAAVKIKKPIKTKFRMPVFNWVALKPNQINGTVFNEIDDERILEDLNVDEFEEIFKTKAQGPAIDLSSSKQKIPQKGSNKVTLLEANRAKNLAITLRKAGKTADEICKAIHVFDLKTLPVDFVECLMRFLPTENEVKVLRLYERERKPLENLSDEDRFMMQFSKIERLMQKMTIMAFIGNFAESIQMLTPQLHAIIAASVSIKSSQKLKKILEIILALGNYMNSSKRGAVYGFKLQSLDLLLDTKSTDRKQTLLHYISNVVKEKYHQVSLFYNELHYVEKAAAVSLENVLLDVKELQRGMDLTKREYTMHDHNTLLKEFILNNEGKLKKLQDDAKIAQDAFDDVVKYFGENPKTTPPSVFFPVFVRFVKAYKQAEEENELRKKQEQALMEKLLEQEALMEQQDPKSPSHKSKRQQQELIAELRRRQVKDNRHVYEGKDGAIEDIITVLKTVPFTARTAKRGSRFFCEPVLTEEYHY",
# BTBD9_HUMAN
"MSNSHPLRPFTAVGEIDHVHILSEHIGALLIGEEYGDVTFVVEKKRFPAHRVILAARCQYFRALLYGGMRESQPEAEIPLQDTTAEAFTMLLKYIYTGRATLTDEKEEVLLDFLSLAHKYGFPELEDSTSEYLCTILNIQNVCMTFDVASLYSLPKLTCMCCMFMDRNAQEVLSSEGFLSLSKTALLNIVLRDSFAAPEKDIFLALLNWCKHNSKENHAEIMQAVRLPLMSLTELLNVVRPSGLLSPDAILDAIKVRSESRDMDLNYRGMLIPEENIATMKYGAQVVKGELKSALLDGDTQNYDLDHGFSRHPIDDDCRSGIEIKLGQPSIINHIRILLWDRDSRSYSYFIEVSMDELDWVRVIDHSQYLCRSWQKLYFPARVCRYIRIVGTHNTVNKIFHIVAFECMFTNKTFTLEKGLIVPMENVATIADCASVIEGVSRSRNALLNGDTKNYDWDSGYTCHQLGSGAIVVQLAQPYMIGSIRLLLWDCDDRSYSYYVEVSTNQQQWTMVADRTKVSCKSWQSVTFERQPASFIRIVGTHNTANEVFHCVHFECPEQQSSQKEENSEESGTGDTSLAGQQLDSHALRAPSGSSLPSSPGSNSRSPNRQHQ",
# ALS2_HUMAN
"MDSKKRSSTEAEGSKERGLVHIWQAGSFPITPERLPGWGGKTVLQAALGVKHGVLLTEDGEVYSFGTLPWRSGPVEICPSSPILENALVGQYVITVATGSFHSGAVTDNGVAYMWGENSAGQCAVANQQYVPEPNPVSIADSEASPLLAVRILQLACGEEHTLALSISREIWAWGTGCQLGLITTAFPVTKPQKVEHLAGRVVLQVACGAFHSLALVQCLPSQDLKPVPERCNQCSQLLITMTDKEDHVIISDSHCCPLGVTLTESQAENHASTALSPSTETLDRQEEVFENTLVANDQSVATELNAVSAQITSSDAMSSQQNVMGTTEISSARNIPSYPDTQAVNEYLRKLSDHSVREDSEHGEKPVPSQPLLEEAIPNLHSPPTTSTSALNSLVVSCASAVGVRVAATYEAGALSLKKVMNFYSTTPCETGAQAGSSAIGPEGLKDSREEQVKQESMQGKKSSSLVDIREEETEGGSRRLSLPGLLSQVSPRLLRKAARVKTRTVVLTPTYSGEADALLPSLRTEVWTWGKGKEGQLGHGDVLPRLQPLCVKCLDGKEVIHLEAGGYHSLALTAKSQVYSWGSNTFGQLGHSDFPTTVPRLAKISSENGVWSIAAGRDYSLFLVDTEDFQPGLYYSGRQDPTEGDNLPENHSGSKTPVLLSCSKLGYISRVTAGKDSYLALVDKNIMGYIASLHELATTERRFYSKLSDIKSQILRPLLSLENLGTTTTVQLLQEVASRFSKLCYLIGQHGASLSSFLHGVKEARSLVILKHSSLFLDSYTEYCTSITNFLVMGGFQLLAKPAIDFLNKNQELLQDLSEVNDENTQLMEILNTLFFLPIRRLHNYAKVLLKLATCFEVASPEYQKLQDSSSCYECLALHLGRKRKEAEYTLGFWKTFPGKMTDSLRKPERRLLCESSNRALSLQHAGRFSVNWFILFNDALVHAQFSTHHVFPLATLWAEPLSEEAGGVNGLKITTPEEQFTLISSTPQEKTKWLRAISQAVDQALRGMSDLPPYGSGSSVQRQEPPISRSAKYTFYKDPRLKDATYDGRWLSGKPHGRGVLKWPDGKMYSGMFRNGLEDGYGEYRIPNKAMNKEDHYVGHWKEGKMCGQGVYSYASGEVFEGCFQDNMRHGHGLLRSGKLTSSSPSMFIGQWVMDKKAGYGVFDDITRGEKYMGMWQDDVCQGNGVVVTQFGLYYEGNFHLNKMMGNGVLLSEDDTIYEGEFSDDWTLSGKGTLTMPNGDYIEGYFSGEWGSGIKITGTYFKPSLYESDKDRPKVFRKLGNLAVPADEKWKAVFDECWRQLGCEGPGQGEVWKAWDNIAVALTTSRRQHRDSPEILSRSQTQTLESLEFIPQHVGAFSVEKYDDIRKYLIKACDTPLHPLGRLVETLVAVYRMTYVGVGANRRLLQEAVKEIKSYLKRIFQLVRFLFPELPEEGSTIPLSAPLPTERKSFCTGKSDSRSESPEPGYVVTSSGLLLPVLLPRLYPPLFMLYALDNDREEDIYWECVLRLNKQPDIALLGFLGVQRKFWPATLSILGESKKVLPTTKDACFASAVECLQQISTTFTPSDKLKVIQQTFEEISQSVLASLHEDFLWSMDDLFPVFLYVVLRARIRNLGSEVHLIEDLMDPYLQHGEQGIMFTTLKACYYQIQREKLN",
# VPS35_HUMAN
"MPTTQQSPQDEQEKLLDEAIQAVKVQSFQMKRCLDKNKLMDALKHASNMLGELRTSMLSPKSYYELYMAISDELHYLEVYLTDEFAKGRKVADLYELVQYAGNIIPRLYLLITVGVVYVKSFPQSRKDILKDLVEMCRGVQHPLRGLFLRNYLLQCTRNILPDEGEPTDEETTGDISDSMDFVLLNFAEMNKLWVRMQHQGHSRDREKRERERQELRILVGTNLVRLSQLEGVNVERYKQIVLTGILEQVVNCRDALAQEYLMECIIQVFPDEFHLQTLNPFLRACAELHQNVNVKNIIIALIDRLALFAHREDGPGIPADIKLFDIFSQQVATVIQSRQDMPSEDVVSLQVSLINLAMKCYPDRVDYVDKVLETTVEIFNKLNLEHIATSSAVSKELTRLLKIPVDTYNNILTVLKLKHFHPLFEYFDYESRKSMSCYVLSNVLDYNTEIVSQDQVDSIMNLVSTLIQDQPDQPVEDPDPEDFADEQSLVGRFIHLLRSEDPDQQYLILNTARKHFGAGGNQRIRFTLPPLVFAAYQLAFRYKENSKVDDKWEKKCQKIFSFAHQTISALIKAELAELPLRLFLQGALAAGEIGFENHETVAYEFMSQAFSLYEDEISDSKAQLAAITLIIGTFERMKCFSEENHEPLRTQCALAASKLLKKPDQGRAVSTCAHLFWSGRNTDKNGEELHGGKRVMECLKKALKIANQCMDPSLQVQLFIEILNRYIYFYEKENDAVTIQVLNQLIQKIREDLPNLESSEETEQINKHFHNTLEHLRLRRESPESEGPIYEGLIL",
# MAGI1_HUMAN
"MSKVIQKKNHWTSRVHECTVKRGPQGELGVTVLGGAEHGEFPYVGAVAAVEAAGLPGGGEGPRLGEGELLLEVQGVRVSGLPRYDVLGVIDSCKEAVTFKAVRQGGRLNKDLRHFLNQRFQKGSPDHELQQTIRDNLYRHAVPCTTRSPREGEVPGVDYNFLTVKEFLDLEQSGTLLEVGTYEGNYYGTPKPPSQPVSGKVITTDALHSLQSGSKQSTPKRTKSYNDMQNAGIVHAENEEEDDVPEMNSSFTADSGEQEEHTLQETALPPVNSSIIAAPITDPSQKFPQYLPLSAEDNLGPLPENWEMAYTENGEVYFIDHNTKTTSWLDPRCLNKQQKPLEECEDDEGVHTEELDSELELPAGWEKIEDPVYGIYYVDHINRKTQYENPVLEAKRKKQLEQQQQQQQQQQQQQQQQQQQQTEEWTEDHSALVPPVIPNHPPSNPEPAREVPLQGKPFFTRNPSELKGKFIHTKLRKSSRGFGFTVVGGDEPDEFLQIKSLVLDGPAALDGKMETGDVIVSVNDTCVLGHTHAQVVKIFQSIPIGASVDLELCRGYPLPFDPDDPNTSLVTSVAILDKEPIIVNGQETYDSPASHSSKTGKVNGMKDARPSSPADVASNSSHGYPNDTVSLASSIATQPELITVHIVKGPMGFGFTIADSPGGGGQRVKQIVDSPRCRGLKEGDLIVEVNKKNVQALTHNQVVDMLVECPKGSEVTLLVQRGGLPVPKKSPKSQPLERKDSQNSSQHSVSSHRSLHTASPSHSTQVLPEFPPAEAQAPDQTDSSGQKKPDPFKIWAQSRSMYENRPMSPSPASGLSKGEREREINSTNFGECPIPDYQEQDIFLWRKETGFGFRILGGNEPGEPIYIGHIVPLGAADTDGRLRSGDELICVDGTPVIGKSHQLVVQLMQQAAKQGHVNLTVRRKVVFAVPKTENEVPSPASSHHSSNQPASLTEEKRTPQGSQNSLNTVSSGSGSTSGIGSGGGGGSGVVSTVVQPYDVEIRRGENEGFGFVIVSSVSRPEAGTTFAGNACVAMPHKIGRIIEGSPADRCGKLKVGDRILAVNGCSITNKSHSDIVNLIKEAGNTVTLRIIPGDESSNATLLTNAEKIATITTTHTPSQQGTQETRNTTKPKQESQFEFKAPQATQEQDFYTVELERGAKGFGFSLRGGREYNMDLYVLRLAEDGPAERCGKMRIGDEILEINGETTKNMKHSRAIELIKNGGRRVRLFLKRGDGSVPEYDPSSDRHGPATGPQGVPEVRAGPDRRQHPSLESSYPPDLHKSSPHGEKRAHARDPKGSREYSRQPNEHHTWNGTSRKPDSGACRPKDRAPEGRRDAQAERAAAANGPKRRSPEKRREGTRSADNTLERREKHEKRRDVSPERRRERSPTRRRDGSPSRRRRSLERLLEQRRSPERRRGGSPERRAKSTDRRRARSPERRRERSLDKRNREDRASHREREEANLKQDAGRSSRHPPEQRRRPYKECSTDLSI",
# ERBIN_HUMAN
"MTTKRSLFVRLVPCRCLRGEEETVTTLDYSHCSLEQVPKEIFTFEKTLEELYLDANQIEELPKQLFNCQSLHKLSLPDNDLTTLPASIANLINLRELDVSKNGIQEFPENIKNCKVLTIVEASVNPISKLPDGFSQLLNLTQLYLNDAFLEFLPANFGRLTKLQILELRENQLKMLPKTMNRLTQLERLDLGSNEFTEVPEVLEQLSGLKEFWMDANRLTFIPGFIGSLKQLTYLDVSKNNIEMVEEGISTCENLQDLLLSSNSLQQLPETIGSLKNITTLKIDENQLMYLPDSIGGLISVEELDCSFNEVEALPSSIGQLTNLRTFAADHNYLQQLPPEIGSWKNITVLFLHSNKLETLPEEMGDMQKLKVINLSDNRLKNLPFSFTKLQQLTAMWLSDNQSKPLIPLQKETDSETQKMVLTNYMFPQQPRTEDVMFISDNESFNPSLWEEQRKQRAQVAFECDEDKDEREAPPREGNLKRYPTPYPDELKNMVKTVQTIVHRLKDEETNEDSGRDLKPHEDQQDINKDVGVKTSESTTTVKSKVDEREKYMIGNSVQKISEPEAEISPGSLPVTANMKASENLKHIVNHDDVFEESEELSSDEEMKMAEMRPPLIETSINQPKVVALSNNKKDDTKETDSLSDEVTHNSNQNNSNCSSPSRMSDSVSLNTDSSQDTSLCSPVKQTHIDINSKIRQEDENFNSLLQNGDILNSSTEEKFKAHDKKDFNLPEYDLNVEERLVLIEKSVDSTATADDTHKLDHINMNLNKLITNDTFQPEIMERSKTQDIVLGTSFLSINSKEETEHLENGNKYPNLESVNKVNGHSEETSQSPNRTEPHDSDCSVDLGISKSTEDLSPQKSGPVGSVVKSHSITNMEIGGLKIYDILSDNGPQQPSTTVKITSAVDGKNIVRSKSATLLYDQPLQVFTGSSSSSDLISGTKAIFKFDSNHNPEEPNIIRGPTSGPQSAPQIYGPPQYNIQYSSSAAVKDTLWHSKQNPQIDHASFPPQLLPRSESTENQSYAKHSANMNFSNHNNVRANTAYHLHQRLGPARHGEMWAISPNDRLIPAVTRSTIQRQSSVSSTASVNLGDPGSTRRAQIPEGDYLSYREFHSAGRTPPMMPGSQRPLSARTYSIDGPNASRPQSARPSINEIPERTMSVSDFNYSRTSPSKRPNARVGSEHSLLDPPGKSKVPRDWREQVLRHIEAKKLEKKHPQTSSSGDPCQDGIFISGQQNYSSATLSHKDVPPDSLMKMPLSNGQMGQPLRPQANYSQIHHPPQASVARHPSREQLIDYLMLKVAHQPPYTQPHCSPRQGHELAKQEIRVRVEKDPELGFSISGGVGGRGNPFRPDDDGIFVTRVQPEGPASKLLQPGDKIIQANGYSFINIEHGQAVSLLKTFQNTVELIIVREVSS",
# KS6C1_HUMAN
"MTSYRERSADLARFYTVTEPQRHPRGYTVYKVTARVVSRRNPEDVQEIIVWKRYSDFKKLHKELWQIHKNLFRHSELFPPFAKGIVFGRFDETVIEERRQCAEDLLQFSANIPALYNSKQLEDFFKGGIINDSSELIGPAEAHSDSLIDTFPECSTEGFSSDSDLVSLTVDVDSLAELDDGMASNQNSPIRTFGLNLSSDSSALGAVASDSEQSKTEEERESRSLFPGSLKPKLGKRDYLEKAGELIKLALKKEEEDDYEAASDFYRKGVDLLLEGVQGESSPTRREAVKRRTAEYLMRAESISSLYGKPQLDDVSQPPGSLSSRPLWNLRSPAEELKAFRVLGVIDKVLLVMDTRTEQTFILKGLRKSSEYSRNRKTIIPRCVPNMVCLHKYIISEESVFLVLQHAEGGKLWSYISKFLNRSPEESFDIKEVKKPTLAKVHLQQPTSSPQDSSSFESRGSDGGSMLKALPLKSSLTPSSQDDSNQEDDGQDSSPKWPDSGSSSEEECTTSYLTLCNEYGQEKIEPGSLNEEPFMKTEGNGVDTKAIKSFPAHLAADSDSPSTQLRAHELKFFPNDDPEAVSSPRTSDSLSRSKNSPMEFFRIDSKDSASELLGLDFGEKLYSLKSEPLKPFFTLPDGDSASRSFNTSESKVEFKAQDTISRGSDDSVPVISFKDAAFDDVSGTDEGRPDLLVNLPGELESTREAAAMGPTKFTQTNIGIIENKLLEAPDVLCLRLSTEQCQAHEEKGIEELSDPSGPKSYSITEKHYAQEDPRMLFVAAVDHSSSGDMSLLPSSDPKFQGLGVVESAVTANNTEESLFRICSPLSGANEYIASTDTLKTEEVLLFTDQTDDLAKEEPTSLFQRDSETKGESGLVLEGDKEIHQIFEDLDKKLALASRFYIPEGCIQRWAAEMVVALDALHREGIVCRDLNPNNILLNDRGHIQLTYFSRWSEVEDSCDSDAIERMYCAPEVGAITEETEACDWWSLGAVLFELLTGKTLVECHPAGINTHTTLNMPECVSEEARSLIQQLLQFNPLERLGAGVAGVEDIKSHPFFTPVDWAELMR",
# NEB2_HUMAN
"MMKTEPRGPGGPLRSASPHRSAYEAGIQALKPPDAPGPDEAPKGAHHKKYGSNVHRIKSMFLQMGTTAGPSGEAGGGAGLAEAPRASERGVRLSLPRASSLNENVDHSALLKLGTSVSERVSRFDSKPAPSAQPAPPPHPPSRLQETRKLFERSAPAAAGGDKEAAARRLLRQERAGLQDRKLDVVVRFNGSTEALDKLDADAVSPTVSQLSAVFEKADSRTGLHRGPGLPRAAGVPQVNSKLVSKRSRVFQPPPPPPPAPSGDAPAEKERCPAGQQPPQHRVAPARPPPKPREVRKIKPVEVEESGESEAESAPGEVIQAEVTVHAALENGSTVATAASPAPEEPKAQAAPEKEAAAVAPPERGVGNGRAPDVAPEEVDESKKEDFSEADLVDVSAYSGLGEDSAGSALEEDDEDDEEDGEPPYEPESGCVEIPGLSEEEDPAPSRKIHFSTAPIQVFSTYSNEDYDRRNEDVDPMAASAEYELEKRVERLELFPVELEKDSEGLGISIIGMGAGADMGLEKLGIFVKTVTEGGAAHRDGRIQVNDLLVEVDGTSLVGVTQSFAASVLRNTKGRVRFMIGRERPGEQSEVAQLIQQTLEQERWQREMMEQRYAQYGEDDEETGEYATDEDEELSPTFPGGEMAIEVFELAENEDALSPVDMEPEKLVHKFKELQIKHAVTEAEIQQLKRKLQSLEQEKGRWRVEKAQLEQSVEENKERMEKLEGYWGEAQSLCQAVDEHLRETQAQYQALERKYSKAKRLIKDYQQKEIEFLKKETAQRRVLEESELARKEEMDKLLDKISELEGNLQTLRNSNST",
# RMD3_HUMAN
"MSRLGALGGARAGLGLLLGTAAGLGFLCLLYSQRWKRTQRHGRSQSLPNSLDYTQTSDPGRHVMLLRAVPGGAGDASVLPSLPREGQEKVLDRLDFVLTSLVALRREVEELRSSLRGLAGEIVGEVRCHMEENQRVARRRRFPFVRERSDSTGSSSVYFTASSGATFTDAESEGGYTTANAESDNERDSDKESEDGEDEVSCETVKMGRKDSLDLEEEAASGASSALEAGGSSGLEDVLPLLQQADELHRGDEQGKREGFQLLLNNKLVYGSRQDFLWRLARAYSDMCELTEEVSEKKSYALDGKEEAEAALEKGDESADCHLWYAVLCGQLAEHESIQRRIQSGFSFKEHVDKAIALQPENPMAHFLLGRWCYQVSHLSWLEKKTATALLESPLSATVEDALQSFLKAEELQPGFSKAGRVYISKCYRELGKNSEARWWMKLALELPDVTKEDLAIQKDLEELEVILRD",
# SCN2A_HUMAN
"MAQSVLVPPGPDSFRFFTRESLAAIEQRIAEEKAKRPKQERKDEDDENGPKPNSDLEAGKSLPFIYGDIPPEMVSVPLEDLDPYYINKKTFIVLNKGKAISRFSATPALYILTPFNPIRKLAIKILVHSLFNMLIMCTILTNCVFMTMSNPPDWTKNVEYTFTGIYTFESLIKILARGFCLEDFTFLRDPWNWLDFTVITFAYVTEFVDLGNVSALRTFRVLRALKTISVIPGLKTIVGALIQSVKKLSDVMILTVFCLSVFALIGLQLFMGNLRNKCLQWPPDNSSFEINITSFFNNSLDGNGTTFNRTVSIFNWDEYIEDKSHFYFLEGQNDALLCGNSSDAGQCPEGYICVKAGRNPNYGYTSFDTFSWAFLSLFRLMTQDFWENLYQLTLRAAGKTYMIFFVLVIFLGSFYLINLILAVVAMAYEEQNQATLEEAEQKEAEFQQMLEQLKKQQEEAQAAAAAASAESRDFSGAGGIGVFSESSSVASKLSSKSEKELKNRRKKKKQKEQSGEEEKNDRVRKSESEDSIRRKGFRFSLEGSRLTYEKRFSSPHQSLLSIRGSLFSPRRNSRASLFSFRGRAKDIGSENDFADDEHSTFEDNDSRRDSLFVPHRHGERRHSNVSQASRASRVLPILPMNGKMHSAVDCNGVVSLVGGPSTLTSAGQLLPEGTTTETEIRKRRSSSYHVSMDLLEDPTSRQRAMSIASILTNTMEELEESRQKCPPCWYKFANMCLIWDCCKPWLKVKHLVNLVVMDPFVDLAITICIVLNTLFMAMEHYPMTEQFSSVLSVGNLVFTGIFTAEMFLKIIAMDPYYYFQEGWNIFDGFIVSLSLMELGLANVEGLSVLRSFRLLRVFKLAKSWPTLNMLIKIIGNSVGALGNLTLVLAIIVFIFAVVGMQLFGKSYKECVCKISNDCELPRWHMHDFFHSFLIVFRVLCGEWIETMWDCMEVAGQTMCLTVFMMVMVIGNLVVLNLFLALLLSSFSSDNLAATDDDNEMNNLQIAVGRMQKGIDFVKRKIREFIQKAFVRKQKALDEIKPLEDLNNKKDSCISNHTTIEIGKDLNYLKDGNGTTSGIGSSVEKYVVDESDYMSFINNPSLTVTVPIAVGESDFENLNTEEFSSESDMEESKEKLNATSSSEGSTVDIGAPAEGEQPEVEPEESLEPEACFTEDCVRKFKCCQISIEEGKGKLWWNLRKTCYKIVEHNWFETFIVFMILLSSGALAFEDIYIEQRKTIKTMLEYADKVFTYIFILEMLLKWVAYGFQVYFTNAWCWLDFLIVDVSLVSLTANALGYSELGAIKSLRTLRALRPLRALSRFEGMRVVVNALLGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFYHCINYTTGEMFDVSVVNNYSECKALIESNQTARWKNVKVNFDNVGLGYLSLLQVATFKGWMDIMYAAVDSRNVELQPKYEDNLYMYLYFVIFIIFGSFFTLNLFIGVIIDNFNQQKKKFGGQDIFMTEEQKKYYNAMKKLGSKKPQKPIPRPANKFQGMVFDFVTKQVFDISIMILICLNMVTMMVETDDQSQEMTNILYWINLVFIVLFTGECVLKLISLRYYYFTIGWNIFDFVVVILSIVGMFLAELIEKYFVSPTLFRVIRLARIGRILRLIKGAKGIRTLLFALMMSLPALFNIGLLLFLVMFIYAIFGMSNFAYVKREVGIDDMFNFETFGNSMICLFQITTSAGWDGLLAPILNSGPPDCDPDKDHPGSSVKGDCGNPSVGIFFFVSYIIISFLVVVNMYIAVILENFSVATEESAEPLSEDDFEMFYEVWEKFDPDATQFIEFAKLSDFADALDPPLLIAKPNKVQLIAMDLPMVSGDRIHCLDILFAFTKRVLGESGEMDALRIQMEERFMASNPSKVSYEPITTTLKRKQEEVSAIIIQRAYRRYLLKQKVKKVSSIYKKDKGKECDGTPIKEDTLIDKLNENSTPEKTDMTPSTTSPPSYDSVTKPEKEKFEKDKSEKEDKGKDIRESKK",
# DCE1_HUMAN
"MASSTPSSSATSSNAGADPNTTNLRPTTYDTWCGVAHGCTRKLGLKICGFLQRTNSLEEKSRLVSAFKERQSSKNLLSCENSDRDARFRRTETDFSNLFARDLLPAKNGEEQTVQFLLEVVDILLNYVRKTFDRSTKVLDFHHPHQLLEGMEGFNLELSDHPESLEQILVDCRDTLKYGVRTGHPRFFNQLSTGLDIIGLAGEWLTSTANTNMFTYEIAPVFVLMEQITLKKMREIVGWSSKDGDGIFSPGGAISNMYSIMAARYKYFPEVKTKGMAAVPKLVLFTSEQSHYSIKKAGAALGFGTDNVILIKCNERGKIIPADFEAKILEAKQKGYVPFYVNATAGTTVYGAFDPIQEIADICEKYNLWLHVDAAWGGGLLMSRKHRHKLNGIERANSVTWNPHKMMGVLLQCSAILVKEKGILQGCNQMCAGYLFQPDKQYDVSYDTGDKAIQCGRHVDIFKFWLMWKAKGTVGFENQINKCLELAEYLYAKIKNREEFEMVFNGEPEHTNVCFWYIPQSLRGVPDSPQRREKLHKVAPKIKALMMESGTTMVGYQPQGDKANFFRMVISNPAATQSDIDFLIEEIERLGQDL",
# CYH2_HUMAN
"MEDGVYEPPDLTPEERMELENIRRRKQELLVEIQRLREELSEAMSEVEGLEANEGSKTLQRNRKMAMGRKKFNMDPKKGIQFLVENELLQNTPEEIARFLYKGEGLNKTAIGDYLGEREELNLAVLHAFVDLHEFTDLNLVQALRQFLWSFRLPGEAQKIDRMMEAFAQRYCLCNPGVFQSTDTCYVLSFAVIMLNTSLHNPNVRDKPGLERFVAMNRGINEGGDLPEELLRNLYDSIRNEPFKIPEDDGNDLTHTFFNPDREGWLLKLGGGRVKTWKRRWFILTDNCLYYFEYTTDKEPRGIIPLENLSIREVDDPRKPNCFELYIPNNKGQLIKACKTEADGRVVEGNHMVYRISAPTQEEKDEWIKSIQAAVSVDPFYEMLAARKKRISVKKKQEQP",
# NELL2_HUMAN
"MESRVLLRTFCLIFGLGAVWGLGVDPSLQIDVLTELELGESTTGVRQVPGLHNGTKAFLFQDTPRSIKASTATAEQFFQKLRNKHEFTILVTLKQTHLNSGVILSIHHLDHRYLELESSGHRNEVRLHYRSGSHRPHTEVFPYILADDKWHKLSLAISASHLILHIDCNKIYERVVEKPSTDLPLGTTFWLGQRNNAHGYFKGIMQDVQLLVMPQGFIAQCPDLNRTCPTCNDFHGLVQKIMELQDILAKTSAKLSRAEQRMNRLDQCYCERTCTMKGTTYREFESWIDGCKNCTCLNGTIQCETLICPNPDCPLKSALAYVDGKCCKECKSICQFQGRTYFEGERNTVYSSSGVCVLYECKDQTMKLVESSGCPALDCPESHQITLSHSCCKVCKGYDFCSERHNCMENSICRNLNDRAVCSCRDGFRALREDNAYCEDIDECAEGRHYCRENTMCVNTPGSFMCICKTGYIRIDDYSCTEHDECITNQHNCDENALCFNTVGGHNCVCKPGYTGNGTTCKAFCKDGCRNGGACIAANVCACPQGFTGPSCETDIDECSDGFVQCDSRANCINLPGWYHCECRDGYHDNGMFSPSGESCEDIDECGTGRHSCANDTICFNLDGGYDCRCPHGKNCTGDCIHDGKVKHNGQIWVLENDRCSVCSCQNGFVMCRRMVCDCENPTVDLFCCPECDPRLSSQCLHQNGETLYNSGDTWVQNCQQCRCLQGEVDCWPLPCPDVECEFSILPENECCPRCVTDPCQADTIRNDITKTCLDEMNVVRFTGSSWIKHGTECTLCQCKNGHICCSVDPQCLQEL",
# AGAP2_HUMAN
"MSRGAGALQRRTTTYLISLTLVKLESVPPPPPSPSAAAAGAAGARGSETGDPGSPRGAEEPGKKRHERLFHRQDALWISTSSAGTGGAEPPALSPAPASPARPVSPAPGRRLSLWAVPPGPPLSGGLSPDPKPGGAPTSSRRPLLSSPSWGGPEPEGRAGGGIPGSSSPHPGTGSRRLKVAPPPPAPKPCKTVTTSGAKAGGGKGAGSRLSWPESEGKPRVKGSKSSAGTGASVSAAATAAAAGGGGSTASTSGGVGAGAGARGKLSPRKGKSKTLDNSDLHPGPPAGSPPPLTLPPTPSPATAVTAASAQPPGPAPPITLEPPAPGLKRGREGGRASTRDRKMLKFISGIFTKSTGGPPGSGPLPGPPSLSSGSGSRELLGAELRASPKAVINSQEWTLSRSIPELRLGVLGDARSGKSSLIHRFLTGSYQVLEKTESEQYKKEMLVDGQTHLVLIREEAGAPDAKFSGWADAVIFVFSLEDENSFQAVSRLHGQLSSLRGEGRGGLALALVGTQDRISASSPRVVGDARARALCADMKRCSYYETCATYGLNVDRVFQEVAQKVVTLRKQQQLLAACKSLPSSPSHSAASTPVAGQASNGGHTSDYSSSLPSSPNVGHRELRAEAAAVAGLSTPGSLHRAAKRRTSLFANRRGSDSEKRSLDSRGETTGSGRAIPIKQSFLLKRSGNSLNKEWKKKYVTLSSNGFLLYHPSINDYIHSTHGKEMDLLRTTVKVPGKRPPRAISAFGPSASINGLVKDMSTVQMGEGLEATTPMPSPSPSPSSLQPPPDQTSKHLLKPDRNLARALSTDCTPSGDLSPLSREPPPSPMVKKQRRKKLTTPSKTEGSAGQAEAKRKMWKLKSFGSLRNIYKAEENFEFLIVSSTGQTWHFEAASFEERDAWVQAIESQILASLQCCESSKVKLRTDSQSEAVAIQAIRNAKGNSICVDCGAPNPTWASLNLGALICIECSGIHRNLGTHLSRVRSLDLDDWPRELTLVLTAIGNDTANRVWESDTRGRAKPSRDSSREERESWIRAKYEQLLFLAPLSTSEEPLGRQLWAAVQAQDVATVLLLLAHARHGPLDTSVEDPQLRSPLHLAAELAHVVITQLLLWYGADVAARDAQGRTALFYARQAGSQLCADILLQHGCPGEGGSAATTPSAATTPSITATPSPRRRSSAASVGRADAPVALV",
# PARK7_HUMAN
"MASKRALVILAKGAEEMETVIPVDVMRRAGIKVTVAGLAGKDPVQCSRDVVICPDASLEDAKKEGPYDVVVLPGGNLGAQNLSESAAVKEILKEQENRKGLIAAICAGPTALLAHEIGFGSKVTTHPLAKDKMMNGGHYTYSENRVEKDGLILTSRGPGTSFEFALAIVEALNGKEVAAQVKAPLVLKD",
# S1PR3_HUMAN
"MATALPPRLQPVRGNETLREHYQYVGKLAGRLKEASEGSTLTTVLFLVICSFIVLENLMVLIAIWKNNKFHNRMYFFIGNLALCDLLAGIAYKVNILMSGKKTFSLSPTVWFLREGSMFVALGASTCSLLAIAIERHLTMIKMRPYDANKRHRVFLLIGMCWLIAFTLGALPILGWNCLHNLPDCSTILPLYSKKYIAFCISIFTAILVTIVILYARIYFLVKSSSRKVANHNNSERSMALLRTVVIVVSVFIACWSPLFILFLIDVACRVQACPILFKAQWFIVLAVLNSAMNPVIYTLASKEMRRAFFRLVCNCLVRGRGARASPIQPALDPSRSKSSSSNNSSHSPKVKEDLPHTAPSSCIMDKNAALQNGIFCN",
# SORT_HUMAN
"MERPWGAADGLSRWPHGLGLLLLLQLLPPSTLSQDRLDAPPPPAAPLPRWSGPIGVSWGLRAAAAGGAFPRGGRWRRSAPGEDEECGRVRDFVAKLANNTHQHVFDDLRGSVSLSWVGDSTGVILVLTTFHVPLVIMTFGQSKLYRSEDYGKNFKDITDLINNTFIRTEFGMAIGPENSGKVVLTAEVSGGSRGGRIFRSSDFAKNFVQTDLPFHPLTQMMYSPQNSDYLLALSTENGLWVSKNFGGKWEEIHKAVCLAKWGSDNTIFFTTYANGSCKADLGALELWRTSDLGKSFKTIGVKIYSFGLGGRFLFASVMADKDTTRRIHVSTDQGDTWSMAQLPSVGQEQFYSILAANDDMVFMHVDEPGDTGFGTIFTSDDRGIVYSKSLDRHLYTTTGGETDFTNVTSLRGVYITSVLSEDNSIQTMITFDQGGRWTHLRKPENSECDATAKNKNECSLHIHASYSISQKLNVPMAPLSEPNAVGIVIAHGSVGDAISVMVPDVYISDDGGYSWTKMLEGPHYYTILDSGGIIVAIEHSSRPINVIKFSTDEGQCWQTYTFTRDPIYFTGLASEPGARSMNISIWGFTESFLTSQWVSYTIDFKDILERNCEEKDYTIWLAHSTDPEDYEDGCILGYKEQFLRLRKSSVCQNGRDYVVTKQPSICLCSLEDFLCDFGYYRPENDSKCVEQPELKGHDLEFCLYGREEHLTTNGYRKIPGDKCQGGVNPVREVKDLKKKCTSNFLSPEKQNSKSNSVPIILAIVGLMLVTVVAGVLIVKKYVCGGRFLVHRYSVLQQHAEANGVDGVDALDTASHTNKSGYHDDSDEDLLE",
# GPER1_HUMAN
"MDVTSQARGVGLEMYPGTAQPAAPNTTSPELNLSHPLLGTALANGTGELSEHQQYVIGLFLSCLYTIFLFPIGFVGNILILVVNISFREKMTIPDLYFINLAVADLILVADSLIEVFNLHERYYDIAVLCTFMSLFLQVNMYSSVFFLTWMSFDRYIALARAMRCSLFRTKHHARLSCGLIWMASVSATLVPFTAVHLQHTDEACFCFADVREVQWLEVTLGFIVPFAIIGLCYSLIVRVLVRAHRHRGLRPRRQKALRMILAVVLVFFVCWLPENVFISVHLLQRTQPGAAPCKQSFRHAHPLTGHIVNLAAFSNSCLNPLIYSFLGETFRDKLRLYIEQKTNLPALNRFCHAALKAVIPDSTEQSDVRFSSAV",
# P2RX4_HUMAN
"MAGCCAALAAFLFEYDTPRIVLIRSRKVGLMNRAVQLLILAYVIGWVFVWEKGYQETDSVVSSVTTKVKGVAVTNTSKLGFRIWDVADYVIPAQEENSLFVMTNVILTMNQTQGLCPEIPDATTVCKSDASCTAGSAGTHSNGVSTGRCVAFNGSVKTCEVAAWCPVEDDTHVPQPAFLKAAENFTLLVKNNIWYPKFNFSKRNILPNITTTYLKSCIYDAKTDPFCPIFRLGKIVENAGHSFQDMAVEGGIMGIQVNWDCNLDRAASLCLPRYSFRRLDTRDVEHNVSPGYNFRFAKYYRDLAGNEQRTLIKAYGIRFDIIVFGKAGKFDIIPTMINIGSGLALLGMATVLCDIIVLYCMKKRLYYREKKYKYVEDYEQGLASELDQ",
# P2RX7_HUMAN
"MPACCSCSDVFQYETNKVTRIQSMNYGTIKWFFHVIIFSYVCFALVSDKLYQRKEPVISSVHTKVKGIAEVKEEIVENGVKKLVHSVFDTADYTFPLQGNSFFVMTNFLKTEGQEQRLCPEYPTRRTLCSSDRGCKKGWMDPQSKGIQTGRCVVYEGNQKTCEVSAWCPIEAVEEAPRPALLNSAENFTVLIKNNIDFPGHNYTTRNILPGLNITCTFHKTQNPQCPIFRLGDIFRETGDNFSDVAIQGGIMGIEIYWDCNLDRWFHHCRPKYSFRRLDDKTTNVSLYPGYNFRYAKYYKENNVEKRTLIKVFGIRFDILVFGTGGKFDIIQLVVYIGSTLSYFGLAAVFIDFLIDTYSSNCCRSHIYPWCKCCQPCVVNEYYYRKKCESIVEPKPTLKYVSFVDESHIRMVNQQLLGRSLQDVKGQEVPRPAMDFTDLSRLPLALHDTPPIPGQPEEIQLLRKEATPRSRDSPVWCQCGSCLPSQLPESHRCLEELCCRKKPGACITTSELFRKLVLSRHVLQFLLLYQEPLLALDVDSTNSRLRHCAYRCYATWRFGSQDMADFAILPSCCRWRIRKEFPKSEGQYSGFKSPY",
# PHB2_HUMAN
"MAQNLKDLAGRLPAGPRGMGTALKLLLGAGAVAYGVRESVFTVEGGHRAIFFNRIGGVQQDTILAEGLHFRIPWFQYPIIYDIRARPRKISSPTGSKDLQMVNISLRVLSRPNAQELPSMYQRLGLDYEERVLPSIVNEVLKSVVAKFNASQLITQRAQVSLLIRRELTERAKDFSLILDDVAITELSFSREYTAAVEAKQVAQQEAQRAQFLVEKAKQEQRQKIVQAEGEAEAAKMLGEALSKNPGYIKLRKIRAAQNISKTIATSQNRIYLTADNLVLNLQDESFTRGSDSLIKGKK",
# KIF2C_HUMAN
"MAMDSSLQARLFPGLAIKIQRSNGLIHSANVRTVNLEKSCVSVEWAEGGATKGKEIDFDDVAAINPELLQLLPLHPKDNLPLQENVTIQKQKRRSVNSKIPAPKESLRSRSTRMSTVSELRITAQENDMEVELPAAANSRKQFSVPPAPTRPSCPAVAEIPLRMVSEEMEEQVHSIRGSSSANPVNSVRRKSCLVKEVEKMKNKREEKKAQNSEMRMKRAQEYDSSFPNWEFARMIKEFRATLECHPLTMTDPIEEHRICVCVRKRPLNKQELAKKEIDVISIPSKCLLLVHEPKLKVDLTKYLENQAFCFDFAFDETASNEVVYRFTARPLVQTIFEGGKATCFAYGQTGSGKTHTMGGDLSGKAQNASKGIYAMASRDVFLLKNQPCYRKLGLEVYVTFFEIYNGKLFDLLNKKAKLRVLEDGKQQVQVVGLQEHLVNSADDVIKMIDMGSACRTSGQTFANSNSSRSHACFQIILRAKGRMHGKFSLVDLAGNERGADTSSADRQTRMEGAEINKSLLALKECIRALGQNKAHTPFRESKLTQVLRDSFIGENSRTCMIATISPGISSCEYTLNTLRYADRVKELSPHSGPSGEQLIQMETEEMEACSNGALIPGNLSKEEEELSSQMSSFNEAMTQIRELEEKAMEELKEIIQQGPDWLELSEMTEQPDYDLETFVNKAESALAQQAKHFSALRDVIKALRLAMQLEEQASRQISSKKRPQ",
# MGLL_HUMAN
"MPEESSPRRTPQSIPYQDLPHLVNADGQYLFCRYWKPTGTPKALIFVSHGAGEHSGRYEELARMLMGLDLLVFAHDHVGHGQSEGERMVVSDFHVFVRDVLQHVDSMQKDYPGLPVFLLGHSMGGAIAILTAAERPGHFAGMVLISPLVLANPESATTFKVLAAKVLNLVLPNLSLGPIDSSVLSRNKTEVDIYNSDPLICRAGLKVCFGIQLLNAVSRVERALPKLTVPFLLLQGSADRLCDSKGAYLLMELAKSQDKTLKIYEGAYHVLHKELPEVTNSVFHEINMWVSQRTATAGTASPP",
# FEZ1_HUMAN
"MEAPLVSLDEEFEDLRPSCSEDPEEKPQCFYGSSPHHLEDPSLSELENFSSEIISFKSMEDLVNEFDEKLNVCFRNYNAKTENLAPVKNQLQIQEEEETLQDEEVWDALTDNYIPSLSEDWRDPNIEALNGNCSDTEIHEKEEEEFNEKSENDSGINEEPLLTADQVIEEIEEMMQNSPDPEEEEEVLEEEDGGETSSQADSVLLQEMQALTQTFNNNWSYEGLRHMSGSELTELLDQVEGAIRDFSEELVQQLARRDELEFEKEVKNSFITVLIEVQNKQKEQRELMKKRRKEKGLSLQSSRIEKGNQMPLKRFSMEGISNILQSGIRQTFGSSGTDKQYLNTVIPYEKKASPPSVEDLQMLTNILFAMKEDNEKVPTLLTDYILKVLCPT",
# SEPT5_HUMAN
"MSTGLRYKSKLATPEDKQDIDKQYVGFATLPNQVHRKSVKKGFDFTLMVAGESGLGKSTLVHSLFLTDLYKDRKLLSAEERISQTVEILKHTVDIEEKGVKLKLTIVDTPGFGDAVNNTECWKPITDYVDQQFEQYFRDESGLNRKNIQDNRVHCCLYFISPFGHGLRPVDVGFMKALHEKVNIVPLIAKADCLVPSEIRKLKERIREEIDKFGIHVYQFPECDSDEDEDFKQQDRELKESAPFAVIGSNTVVEAKGQRVRGRLYPWGIVEVENQAHCDFVKLRNMLIRTHMHDLKDVTCDVHYENYRAHCIQQMTSKLTQDSRMESPIPILPLPTPDAETEKLIRMKDEELRRMQEMLQRMKQQMQDQ",
# SGMR1_HUMAN
"MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLARQYAGLDHELAFSRLIVELRRLHPGHVLPDEELQWVFVNAGGWMGAMCLLHASLSEYVLLFGTALGSRGHSGRYWAEISDTIISGTFHQWREGTTKSEVFYPGETVVHGPGEATAVEWGPNTWMVEYGRGVIPSTLAFALADTVFSTQDFLTLFYTLRSYARGLRLELTTYLFGQDP",
# ZNT3_HUMAN
"MEPSPAAGGLETTRLVSPRDRGGAGGSLRLKSLFTEPSEPLPEESKPVEMPFHHCHRDPLPPPGLTPERLHARRQLYAACAVCFVFMAGEVVGGYLAHSLAIMTDAAHLLADVGSMMGSLFSLWLSTRPATRTMTFGWHRSETLGALASVVSLWMVTGILLYLAFVRLLHSDYHIEGGAMLLTASIAVCANLLMAFVLHQAGPPHSHGSRGAEYAPLEEGPEEPLPLGNTSVRAAFVHVLGDLLQSFGVLAASILIYFKPQYKAADPISTFLFSICALGSTAPTLRDVLRILMEGTPRNVGFEPVRDTLLSVPGVRATHELHLWALTLTYHVASAHLAIDSTADPEAVLAEASSRLYSRFGFSSCTLQVEQYQPEMAQCLRCQEPPQA",
# ROAA_HUMAN
"MSEAGEEQPMETTGATENGHEAVPEASRGRGWTGAAAGAGGATAAPPSGNQNGAEGDQINASKNEEDAGKMFVGGLSWDTSKKDLKDYFTKFGEVVDCTIKMDPNTGRSRGFGFILFKDAASVEKVLDQKEHRLDGRVIDPKKAMAMKKDPVKKIFVGGLNPESPTEEKIREYFGEFGEIEAIELPMDPKLNKRRGFVFITFKEEEPVKKVLEKKFHTVSGSKCEIKVAQPKEVYQQQQYGSGGRGNRNRGNRGSGGGGGGGGQSQSWNQGYGNYWNQGYGYQQGYGPGYGGYDYSPYGYYGYGPGYDYSQGSTNYGKSQRRGGHQNNYKPY",
# SNAG_HUMAN
"MAAQKINEGLEHLAKAEKYLKTGFLKWKPDYDSAASEYGKAAVAFKNAKQFEQAKDACLREAVAHENNRALFHAAKAYEQAGMMLKEMQKLPEAVQLIEKASMMYLENGTPDTAAMALERAGKLIENVDPEKAVQLYQQTANVFENEERLRQAVELLGKASRLLVRGRRFDEAALSIQKEKNIYKEIENYPTCYKKTIAQVLVHLHRNDYVAAERCVRESYSIPGFNGSEDCAALEQLLEGYDQQDQDQVSDVCNSPLFKYMDNDYAKLGLSLVVPGGGIKKKSPATPQAKPDGVTATAADEEEDEYSGGLC",
# APBA2_HUMAN
"MAHRKLESVGSGMLDHRVRPGPVPHSQEPESEDMELPLEGYVPEGLELAALRPESPAPEEQECHNHSPDGDSSSDYVNNTSEEEDYDEGLPEEEEGITYYIRYCPEDDSYLEGMDCNGEEYLAHSAHPVDTDECQEAVEEWTDSAGPHPHGHEAEGSQDYPDGQLPIPEDEPSVLEAHDQEEDGHYCASKEGYQDYYPEEANGNTGASPYRLRRGDGDLEDQEEDIDQIVAEIKMSLSMTSITSASEASPEHGPEPGPEDSVEACPPIKASCSPSRHEARPKSLNLLPEAKHPGDPQRGFKPKTRTPEERLKWPHEQVCNGLEQPRKQQRSDLNGPVDNNNIPETKKVASFPSFVAVPGPCEPEDLIDGIIFAANYLGSTQLLSERNPSKNIRMMQAQEAVSRVKRMQKAAKIKKKANSEGDAQTLTEVDLFISTQRIKVLNADTQETMMDHALRTISYIADIGNIVVLMARRRMPRSASQDCIETTPGAQEGKKQYKMICHVFESEDAQLIAQSIGQAFSVAYQEFLRANGINPEDLSQKEYSDIINTQEMYNDDLIHFSNSENCKELQLEKHKGEILGVVVVESGWGSILPTVILANMMNGGPAARSGKLSIGDQIMSINGTSLVGLPLATCQGIIKGLKNQTQVKLNIVSCPPVTTVLIKRPDLKYQLGFSVQNGIICSLMRGGIAERGGVRVGHRIIEINGQSVVATAHEKIVQALSNSVGEIHMKTMPAAMFRLLTGQETPLYI",
# NOE1_HUMAN
"MSVPLLKIGVVLSTMAMITNWMSQTLPSLVGLNTTKLSAAGGGTLDRSTGVLPTNPEESWQVYSSAQDSEGRCICTVVAPQQTMCSRDARTKQLRQLLEKVQNMSQSIEVLDRRTQRDLQYVEKMENQMKGLESKFKQVEESHKQHLARQFKAIKAKMDELRPLIPVLEEYKADAKLVLQFKEEVQNLTSVLNELQEEIGAYDYDELQSRVSNLEERLRACMQKLACGKLTGISDPVTVKTSGSRFGSWMTDPLAPEGDNRVWYMDGYHNNRFVREYKSMVDFMNTDNFTSHRLPHPWSGTGQVVYNGSIYFNKFQSHIIIRFDLKTETILKTRSLDYAGYNNMYHYAWGGHSDIDLMVDESGLWAVYATNQNAGNIVVSRLDPVSLQTLQTWNTSYPKRSAGEAFIICGTLYVTNGYSGGTKVHYAYQTNASTYEYIDIPFQNKYSHISMLDYNPKDRALYAWNNGHQILYNVTLFHVIRSDEL",
# S29A1_HUMAN
"MTTSHQPQDRYKAVWLIFFMLGLGTLLPWNFFMTATQYFTNRLDMSQNVSLVTAELSKDAQASAAPAAPLPERNSLSAIFNNVMTLCAMLPLLLFTYLNSFLHQRIPQSVRILGSLVAILLVFLITAILVKVQLDALPFFVITMIKIVLINSFGAILQGSLFGLAGLLPASYTAPIMSGQGLAGFFASVAMICAIASGSELSESAFGYFITACAVIILTIICYLGLPRLEFYRYYQQLKLEGPGEQETKLDLISKGEEPRAGKEESGVSVSNSQPTNESHSIKAILKNISVLAFSVCFIFTITIGMFPAVTVEVKSSIAGSSTWERYFIPVSCFLTFNIFDWLGRSLTAVFMWPGKDSRWLPSLVLARLVFVPLLLLCNIKPRRYLTVVFEHDAWFIFFMAAFAFSNGYLASLCMCFGPKKVKPAEAETAGAIMAFFLCLGLALGAVFSFLFRAIV",
# SMO_HUMAN
"MAAARPARGPELPLLGLLLLLLLGDPGRGAASSGNATGPGPRSAGGSARRSAAVTGPPPPLSHCGRAAPCEPLRYNVCLGSVLPYGATSTLLAGDSDSQEEAHGKLVLWSGLRNAPRCWAVIQPLLCAVYMPKCENDRVELPSRTLCQATRGPCAIVERERGWPDFLRCTPDRFPEGCTNEVQNIKFNSSGQCEVPLVRTDNPKSWYEDVEGCGIQCQNPLFTEAEHQDMHSYIAAFGAVTGLCTLFTLATFVADWRNSNRYPAVILFYVNACFFVGSIGWLAQFMDGARREIVCRADGTMRLGEPTSNETLSCVIIFVIVYYALMAGVVWFVVLTYAWHTSFKALGTTYQPLSGKTSYFHLLTWSLPFVLTVAILAVAQVDGDSVSGICFVGYKNYRYRAGFVLAPIGLVLIVGGYFLIRGVMTLFSIKSNHPGLLSEKAASKINETMLRLGIFGFLAFGFVLITFSCHFYDFFNQAEWERSFRDYVLCQANVTIGLPTKQPIPDCEIKNRPSLLVEKINLFAMFGTGIAMSTWVWTKATLLIWRRTWCRLTGQSDDEPKRIKKSKMIAKAFSKRHELLQNPGQELSFSMHTVSHDGPVAGLAFDLNEPSADVSSAWAQHVTKMVARRGAILPQDISVTPVATPVPPEEQANLWLVEAEISPELQKRLGRKKKRRKRKKEVCPLAPPPELHPPAPAPSTIPRLPQLPRQKCLVAAGAWGAGDSCRQGAWTLVSNPFCPEPSPPQDPFLPSAPAPVAWAHGRRQGLGPIHSRTNLMDTELMDADSDF",
# SC6A7_HUMAN
"MKKLQGAHLRKPVTPDLLMTPSDQGDVDLDVDFAAHRGNWTGKLDFLLSCIGYCVGLGNVWRFPYRAYTNGGGAFLVPYFLMLAICGIPLFFLELSLGQFSSLGPLAVWKISPLFKGAGAAMLLIVGLVAIYYNMIIAYVLFYLFASLTSDLPWEHCGNWWNTELCLEHRVSKDGNGALPLNLTCTVSPSEEYWSRYVLHIQGSQGIGSPGEIRWNLCLCLLLAWVIVFLCILKGVKSSGKVVYFTATFPYLILLMLLVRGVTLPGAWKGIQFYLTPQFHHLLSSKVWIEAALQIFYSLGVGFGGLLTFASYNTFHQNIYRDTFIVTLGNAITSILAGFAIFSVLGYMSQELGVPVDQVAKAGPGLAFVVYPQAMTMLPLSPFWSFLFFFMLLTLGLDSQFAFLETIVTAVTDEFPYYLRPKKAVFSGLICVAMYLMGLILTTDGGMYWLVLLDDYSASFGLMVVVITTCLAVTRVYGIQRFCRDIHMMLGFKPGLYFRACWLFLSPATLLALMVYSIVKYQPSEYGSYRFPPWAELLGILMGLLSCLMIPAGMLVAVLREEGSLWERLQQASRPAMDWGPSLEENRTGMYVATLAGSQSPKPLMVHMRKYGGITSFENTAIEVDREIAEEEESMM",
# GBRG3_HUMAN
"MAPKLLLLLCLFSGLHARSRKVEEDEYEDSSSNQKWVLAPKSQDTDVTLILNKLLREYDKKLRPDIGIKPTVIDVDIYVNSIGPVSSINMEYQIDIFFAQTWTDSRLRFNSTMKILTLNSNMVGLIWIPDTIFRNSKTAEAHWITTPNQLLRIWNDGKILYTLRLTINAECQLQLHNFPMDEHSCPLIFSSYGYPKEEMIYRWRKNSVEAADQKSWRLYQFDFMGLRNTTEIVTTSAGDYVVMTIYFELSRRMGYFTIQTYIPCILTVVLSWVSFWIKKDATPARTALGITTVLTMTTLSTIARKSLPRVSYVTAMDLFVTVCFLFVFAALMEYATLNYYSSCRKPTTTKKTTSLLHPDSSRWIPERISLQAPSNYSLLDMRPPPTAMITLNNSVYWQEFEDTCVYECLDGKDCQSFFCCYEECKSGSWRKGRIHIDILELDSYSRVFFPTSFLLFNLVYWVGYLYL",
# PRRT1_HUMAN
"MSSEKSGLPDSVPHTSPPPYNAPQPPAEPPAPPPQAAPSSHHHHHHHYHQSGTATLPRLGAGGLASSAATAQRGPSSSATLPRPPHHAPPGPAAGAPPPGCATLPRMPPDPYLQETRFEGPLPPPPPAAAAPPPPAPAQTAQAPGFVVPTHAGTVGTLPLGGYVAPGYPLQLQPCTAYVPVYPVGTPYAGGTPGGTGVTSTLPPPPQGPGLALLEPRRPPHDYMPIAVLTTICCFWPTGIIAIFKAVQVRTALARGDMVSAEIASREARNFSFISLAVGIAAMVLCTILTVVIIIAAQHHENYWDP",
# SH3G1_HUMAN
"MSVAGLKKQFYKASQLVSEKVGGAEGTKLDDDFKEMEKKVDVTSKAVTEVLARTIEYLQPNPASRAKLTMLNTVSKIRGQVKNPGYPQSEGLLGECMIRHGKELGGESNFGDALLDAGESMKRLAEVKDSLDIEVKQNFIDPLQNLCEKDLKEIQHHLKKLEGRRLDFDYKKKRQGKIPDEELRQALEKFEESKEVAETSMHNLLETDIEQVSQLSALVDAQLDYHRQAVQILDELAEKLKRRMREASSRPKREYKPKPREPFDLGEPEQSNGGFPCTTAPKIAASSSFRSSDKPIRTPSRSMPPLDQPSCKALYDFEPENDGELGFHEGDVITLTNQIDENWYEGMLDGQSGFFPLSYVEVLVPLPQ",
# SH3G2_HUMAN
"MSVAGLKKQFHKATQKVSEKVGGAEGTKLDDDFKEMERKVDVTSRAVMEIMTKTIEYLQPNPASRAKLSMINTMSKIRGQEKGPGYPQAEALLAEAMLKFGRELGDDCNFGPALGEVGEAMRELSEVKDSLDIEVKQNFIDPLQNLHDKDLREIQHHLKKLEGRRLDFDYKKKRQGKIPDEELRQALEKFDESKEIAESSMFNLLEMDIEQVSQLSALVQAQLEYHKQAVQILQQVTVRLEERIRQASSQPRREYQPKPRMSLEFPTGDSTQPNGGLSHTGTPKPSGVQMDQPCCRALYDFEPENEGELGFKEGDIITLTNQIDENWYEGMLHGHSGFFPINYVEILVALPH",
# SH3G3_HUMAN
"MSVAGLKKQFHKASQLFSEKISGAEGTKLDDEFLDMERKIDVTNKVVAEILSKTTEYLQPNPAYRAKLGMLNTVSKIRGQVKTTGYPQTEGLLGDCMLKYGKELGEDSTFGNALIEVGESMKLMAEVKDSLDINVKQTFIDPLQLLQDKDLKEIGHHLKKLEGRRLDYDYKKKRVGKIPDEEVRQAVEKFEESKELAERSMFNFLENDVEQVSQLAVFIEAALDYHRQSTEILQELQSKLQMRISAASSVPRREYKPRPVKRSSSELNGVSTTSVVKTTGSNIPMDQPCCRGLYDFEPENQGELGFKEGDIITLTNQIDENWYEGMIHGESGFFPINYVEVIVPLPQ",
# AKAP9_HUMAN
"MEDEERQKKLEAGKAKLAQFRQRKAQSDGQSPSKKQKKKRKTSSSKHDVSAHHDLNIDQSQCNEMYINSSQRVESTVIPESTIMRTLHSGEITSHEQGFSVELESEISTTADDCSSEVNGCSFVMRTGKPTNLLREEEFGVDDSYSEQGAQDSPTHLEMMESELAGKQHEIEELNRELEEMRVTYGTEGLQQLQEFEAAIKQRDGIITQLTANLQQARREKDETMREFLELTEQSQKLQIQFQQLQASETLRNSTHSSTAADLLQAKQQILTHQQQLEEQDHLLEDYQKKKEDFTMQISFLQEKIKVYEMEQDKKVENSNKEEIQEKETIIEELNTKIIEEEKKTLELKDKLTTADKLLGELQEQIVQKNQEIKNMKLELTNSKQKERQSSEEIKQLMGTVEELQKRNHKDSQFETDIVQRMEQETQRKLEQLRAELDEMYGQQIVQMKQELIRQHMAQMEEMKTRHKGEMENALRSYSNITVNEDQIKLMNVAINELNIKLQDTNSQKEKLKEELGLILEEKCALQRQLEDLVEELSFSREQIQRARQTIAEQESKLNEAHKSLSTVEDLKAEIVSASESRKELELKHEAEVTNYKIKLEMLEKEKNAVLDRMAESQEAELERLRTQLLFSHEEELSKLKEDLEIEHRINIEKLKDNLGIHYKQQIDGLQNEMSQKIETMQFEKDNLITKQNQLILEISKLKDLQQSLVNSKSEEMTLQINELQKEIEILRQEEKEKGTLEQEVQELQLKTELLEKQMKEKENDLQEKFAQLEAENSILKDEKKTLEDMLKIHTPVSQEERLIFLDSIKSKSKDSVWEKEIEILIEENEDLKQQCIQLNEEIEKQRNTFSFAEKNFEVNYQELQEEYACLLKVKDDLEDSKNKQELEYKSKLKALNEELHLQRINPTTVKMKSSVFDEDKTFVAETLEMGEVVEKDTTELMEKLEVTKREKLELSQRLSDLSEQLKQKHGEISFLNEEVKSLKQEKEQVSLRCRELEIIINHNRAENVQSCDTQVSSLLDGVVTMTSRGAEGSVSKVNKSFGEESKIMVEDKVSFENMTVGEESKQEQLILDHLPSVTKESSLRATQPSENDKLQKELNVLKSEQNDLRLQMEAQRICLSLVYSTHVDQVREYMENEKDKALCSLKEELIFAQEEKIKELQKIHQLELQTMKTQETGDEGKPLHLLIGKLQKAVSEECSYFLQTLCSVLGEYYTPALKCEVNAEDKENSGDYISENEDPELQDYRYEVQDFQENMHTLLNKVTEEYNKLLVLQTRLSKIWGQQTDGMKLEFGEENLPKEETEFLSIHSQMTNLEDIDVNHKSKLSSLQDLEKTKLEEQVQELESLISSLQQQLKETEQNYEAEIHCLQKRLQAVSESTVPPSLPVDSVVITESDAQRTMYPGSCVKKNIDGTIEFSGEFGVKEETNIVKLLEKQYQEQLEEEVAKVIVSMSIAFAQQTELSRISGGKENTASSKQAHAVCQQEQHYFNEMKLSQDQIGFQTFETVDVKFKEEFKPLSKELGEHGKEILLSNSDPHDIPESKDCVLTISEEMFSKDKTFIVRQSIHDEISVSSMDASRQLMLNEEQLEDMRQELVRQYQEHQQATELLRQAHMRQMERQREDQEQLQEEIKRLNRQLAQRSSIDNENLVSERERVLLEELEALKQLSLAGREKLCCELRNSSTQTQNGNENQGEVEEQTFKEKELDRKPEDVPPEILSNERYALQKANNRLLKILLEVVKTTAAVEETIGRHVLGILDRSSKSQSSASLIWRSEAEASVKSCVHEEHTRVTDESIPSYSGSDMPRNDINMWSKVTEEGTELSQRLVRSGFAGTEIDPENEELMLNISSRLQAAVEKLLEAISETSSQLEHAKVTQTELMRESFRQKQEATESLKCQEELRERLHEESRAREQLAVELSKAEGVIDGYADEKTLFERQIQEKTDIIDRLEQELLCASNRLQELEAEQQQIQEERELLSRQKEAMKAEAGPVEQQLLQETEKLMKEKLEVQCQAEKVRDDLQKQVKALEIDVEEQVSRFIELEQEKNTELMDLRQQNQALEKQLEKMRKFLDEQAIDREHERDVFQQEIQKLEQQLKVVPRFQPISEHQTREVEQLANHLKEKTDKCSELLLSKEQLQRDIQERNEEIEKLEFRVRELEQALLVSADTFQKVEDRKHFGAVEAKPELSLEVQLQAERDAIDRKEKEITNLEEQLEQFREELENKNEEVQQLHMQLEIQKKESTTRLQELEQENKLFKDDMEKLGLAIKESDAMSTQDQHVLFGKFAQIIQEKEVEIDQLNEQVTKLQQQLKITTDNKVIEEKNELIRDLETQIECLMSDQECVKRNREEEIEQLNEVIEKLQQELANIGQKTSMNAHSLSEEADSLKHQLDVVIAEKLALEQQVETANEEMTFMKNVLKETNFKMNQLTQELFSLKRERESVEKIQSIPENSVNVAIDHLSKDKPELEVVLTEDALKSLENQTYFKSFEENGKGSIINLETRLLQLESTVSAKDLELTQCYKQIKDMQEQGQFETEMLQKKIVNLQKIVEEKVAAALVSQIQLEAVQEYAKFCQDNQTISSEPERTNIQNLNQLREDELGSDISALTLRISELESQVVEMHTSLILEKEQVEIAEKNVLEKEKKLLELQKLLEGNEKKQREKEKKRSPQDVEVLKTTTELFHSNEESGFFNELEALRAESVATKAELASYKEKAEKLQEELLVKETNMTSLQKDLSQVRDHLAEAKEKLSILEKEDETEVQESKKACMFEPLPIKLSKSIASQTDGTLKISSSNQTPQILVKNAGIQINLQSECSSEEVTEIISQFTEKIEKMQELHAAEILDMESRHISETETLKREHYVAVQLLKEECGTLKAVIQCLRSKEGSSIPELAHSDAYQTREICSSDSGSDWGQGIYLTHSQGFDIASEGRGEESESATDSFPKKIKGLLRAVHNEGMQVLSLTESPYSDGEDHSIQQVSEPWLEERKAYINTISSLKDLITKMQLQREAEVYDSSQSHESFSDWRGELLLALQQVFLEERSVLLAAFRTELTALGTTDAVGLLNCLEQRIQEQGVEYQAAMECLQKADRRSLLSEIQALHAQMNGRKITLKREQESEKPSQELLEYNIQQKQSQMLEMQVELSSMKDRATELQEQLSSEKMVVAELKSELAQTKLELETTLKAQHKHLKELEAFRLEVKDKTDEVHLLNDTLASEQKKSRELQWALEKEKAKLGRSEERDKEELEDLKFSLESQKQRNLQLNLLLEQQKQLLNESQQKIESQRMLYDAQLSEEQGRNLELQVLLESEKVRIREMSSTLDRERELHAQLQSSDGTGQSRPPLPSEDLLKELQKQLEEKHSRIVELLNETEKYKLDSLQTRQQMEKDRQVHRKTLQTEQEANTEGQKKMHELQSKVEDLQRQLEEKRQQVYKLDLEGQRLQGIMQEFQKQELEREEKRESRRILYQNLNEPTTWSLTSDRTRNWVLQQKIEGETKESNYAKLIEMNGGGTGCNHELEMIRQKLQCVASKLQVLPQKASERLQFETADDEDFIWVQENIDEIILQLQKLTGQQGEEPSLVSPSTSCGSLTERLLRQNAELTGHISQLTEEKNDLRNMVMKLEEQIRWYRQTGAGRDNSSRFSLNGGANIEAIIASEKEVWNREKLTLQKSLKRAEAEVYKLKAELRNDSLLQTLSPDSEHVTLKRIYGKYLRAESFRKALIYQKKYLLLLLGGFQECEDATLALLARMGGQPAFTDLEVITNRPKGFTRFRSAVRVSIAISRMKFLVRRWHRVTGSVSININRDGFGLNQGAEKTDSFYHSSGGLELYGEPRHTTYRSRSDLDYIRSPLPFQNRYPGTPADFNPGSLACSQLQNYDPDRALTDYITRLEALQRRLGTIQSGSTTQFHAGMRR",
# DPYL5_HUMAN
"MLANSASVRILIKGGKVVNDDCTHEADVYIENGIIQQVGRELMIPGGAKVIDATGKLVIPGGIDTSTHFHQTFMNATCVDDFYHGTKAALVGGTTMIIGHVLPDKETSLVDAYEKCRGLADPKVCCDYALHVGITWWAPKVKAEMETLVREKGVNSFQMFMTYKDLYMLRDSELYQVLHACKDIGAIARVHAENGELVAEGAKEALDLGITGPEGIEISRPEELEAEATHRVITIANRTHCPIYLVNVSSISAGDVIAAAKMQGKVVLAETTTAHATLTGLHYYHQDWSHAAAYVTVPPLRLDTNTSTYLMSLLANDTLNIVASDHRPFTTKQKAMGKEDFTKIPHGVSGVQDRMSVIWERGVVGGKMDENRFVAVTSSNAAKLLNLYPRKGRIIPGADADVVVWDPEATKTISASTQVQGGDFNLYENMRCHGVPLVTISRGRVVYENGVFMCAEGTGKFCPLRSFPDTVYKKLVQREKTLKVRGVDRTPYLGDVAVVVHPGKKEMGTPLADTPTRPVTRHGGMRDLHESSFSLSGSQIDDHVPKRASARILAPPGGRSSGIW",
# ARP5L_HUMAN
"MARNTLSSRFRRVDIDEFDENKFVDEQEEAAAAAAEPGPDPSEVDGLLRQGDMLRAFHAALRNSPVNTKNQAVKERAQGVVLKVLTNFKSSEIEQAVQSLDRNGVDLLMKYIYKGFEKPTENSSAVLLQWHEKALAVGGLGSIIRVLTARKTV",
# SENP7_HUMAN
"MDKRKLGRRPSSSEIITEGKRKKSSSDLSEIRKMLNAKPEDVHVQSPLSKFRSSERWTLPLQWERSLRNKVISLDHKNKKHIRGCPVTSKSSPERQLKVMLTNVLWTDLGRKFRKTLPRNDANLCDANKVQSDSLPSTSVDSLETCQKLEPLRQSLNLSERIPRVILTNVLGTELGRKYIRTPPVTEGSLSDTDNLQSEQLSSSSDGSLESYQNLNPHKSCYLSERGSQRSKTVDDNSAKQTAHNKEKRRKDDGISLLISDTQPEDLNSGSRGCDHLEQESRNKDVKYSDSKVELTLISRKTKRRLRNNLPDSQYCTSLDKSTEQTKKQEDDSTISTEFEKPSENYHQDPKLPEEITTKPTKSDFTKLSSLNSQELTLSNATKSASAGSTTETVENSNSIDIVGISSLVEKDENELNTIEKPILRGHNEGNQSLISAEPIVVSSDEEGPVEHKSSEILKLQSKQDRETTNENESTSESALLELPLITCESVQMSSELCPYNPVMENISSIMPSNEMDLQLDFIFTSVYIGKIKGASKGCVTITKKYIKIPFQVSLNEISLLVDTTHLKRFGLWKSKDDNHSKRSHAILFFWVSSDYLQEIQTQLEHSVLSQQSKSSEFIFLELHNPVSQREELKLKDIMTEISIISGELELSYPLSWVQAFPLFQNLSSKESSFIHYYCVSTCSFPAGVAVAEEMKLKSVSQPSNTDAAKPTYTFLQKQSSGCYSLSITSNPDEEWREVRHTGLVQKLIVYPPPPTKGGLGVTNEDLECLEEGEFLNDVIIDFYLKYLILEKASDELVERSHIFSSFFYKCLTRKENNLTEDNPNLSMAQRRHKRVRTWTRHINIFNKDYIFVPVNESSHWYLAVICFPWLEEAVYEDFPQTVSQQSQAQQSQNDNKTIDNDLRTTSTLSLSAEDSQSTESNMSVPKKMCKRPCILILDSLKAASVQNTVQNLREYLEVEWEVKLKTHRQFSKTNMVDLCPKVPKQDNSSDCGVYLLQYVESFFKDPIVNFELPIHLEKWFPRHVIKTKREDIRELILKLHLQQQKGSSS",
# SYT3_HUMAN
"MSGDYEDDLCRRALILVSDLCARVRDADTNDRCQEFNDRIRGYPRGPDADISVSLLSVIVTFCGIVLLGVSLFVSWKLCWVPWRDKGGSAVGGGPLRKDLGPGVGLAGLVGGGGHHLAAGLGGHPLLGGPHHHAHAAHHPPFAELLEPGSLGGSDTPEPSYLDMDSYPEAAAAAVAAGVKPSQTSPELPSEGGAGSGLLLLPPSGGGLPSAQSHQQVTSLAPTTRYPALPRPLTQQTLTSQPDPSSEERPPALPLPLPGGEEKAKLIGQIKPELYQGTGPGGRRSGGGPGSGEAGTGAPCGRISFALRYLYGSDQLVVRILQALDLPAKDSNGFSDPYVKIYLLPDRKKKFQTKVHRKTLNPVFNETFQFSVPLAELAQRKLHFSVYDFDRFSRHDLIGQVVLDNLLELAEQPPDRPLWRDIVEGGSEKADLGELNFSLCYLPTAGRLTVTIIKASNLKAMDLTGFSDPYVKASLISEGRRLKKRKTSIKKNTLNPTYNEALVFDVAPESVENVGLSIAVVDYDCIGHNEVIGVCRVGPDAADPHGREHWAEMLANPRKPVEHWHQLVEEKTVTSFTKGSKGLSEKENSE",
# SGIP1_HUMAN
"MMEGLKKRTRKAFGIRKKEKDTDSTGSPDRDGIQPSPHEPPYNSKAECAREGGKKVSKKSNGAPNGFYAEIDWERYNSPELDEEGYSIRPEEPGSTKGKHFYSSSESEEEEESHKKFNIKIKPLQSKDILKNAATVDELKASIGNIALSPSPVRKSPRRSPGAIKRNLSSEEVARPRRSTPTPELISKKPPDDTTALAPLFGPPLESAFDEQKTEVLLDQPEIWGSGQPINPSMESPKLTRPFPTGTPPPLPPKNVPATPPRTGSPLTIGPGNDQSATEVKIEKLPSINDLDSIFGPVLSPKSVAVNAEEKWVHFSDTSPEHVTPELTPREKVVSPPATPDNPADSPAPGPLGPPGPTGPPGPPGPPRNVLSPLNLEEVQKKVAEQTFIKDDYLETISSPKDFGLGQRATPPPPPPPTYRTVVSSPGPGSGPGPGTTSGASSPARPATPLVPCRSTTPPPPPPRPPSRPKLPPGKPGVGDVSRPFSPPIHSSSPPPIAPLARAESTSSISSTNSLSAATTPTVENEQPSLVWFDRGKFYLTFEGSSRGPSPLTMGAQDTLPVAAAFTETVNAYFKGADPSKCIVKITGEMVLSFPAGITRHFANNPSPAALTFRVINFSRLEHVLPNPQLLCCDNTQNDANTKEFWVNMPNLMTHLKKVSEQKPQATYYNVDMLKYQVSAQGIQSTPLNLAVNWRCEPSSTDLRIDYKYNTDAMTTAVALNNVQFLVPIDGGVTKLQAVLPPAVWNAEQQRILWKIPDISQKSENGGVGSLLARFQLSEGPSKPSPLVVQFTSEGSTLSGCDIELVGAGYRFSLIKKRFAAGKYLADN",
# PSD2_HUMAN
"MEEDKLLSAVPEEGDATRDPGPEPEEEPGVRNGMASEGLNSSLCSPGHERRGTPADTEEPTKDPDVAFHGLSLGLSLTNGLALGPDLNILEDSAESRPWRAGVLAEGDNASRSLYPDAEDPQLGLDGPGEPDVRDGFSATFEKILESELLRGTQYSSLDSLDGLSLTDESDSCVSFEAPLTPLIQQRARDSPEPGAGLGIGDMAFEGDMGAAGGDGELGSPLRRSISSSRSENVLSRLSLMAMPNGFHEDGPQGPGGDEDDDEEDTDKLLNSASDPSLKDGLSDSDSELSSSEGLEPGSADPLANGCQGVSEAAHRLARRLYHLEGFQRCDVARQLGKNNEFSRLVAGEYLSFFDFSGLTLDGALRTFLKAFPLMGETQERERVLTHFSRRYCQCNPDDSTSEDGIHTLTCALMLLNTDLHGHNIGKKMSCQQFIANLDQLNDGQDFAKDLLKTLYNSIKNEKLEWAIDEDELRKSLSELVDDKFGTGTKKVTRILDGGNPFLDVPQALSATTYKHGVLTRKTHADMDGKRTPRGRRGWKKFYAVLKGTILYLQKDEYRPDKALSEGDLKNAIRVHHALATRASDYSKKSNVLKLKTADWRVFLFQAPSKEEMLSWILRINLVAAIFSAPAFPAAVSSMKKFCRPLLPSCTTRLCQEEQLRSHENKLRQLTAELAEHRCHPVERGIKSKEAEEYRLKEHYLTFEKSRYETYIHLLAMKIKVGSDDLERIEARLATLEGDDPSLRKTHSSPALSQGHVTGSKTTKDATGPDT",
# CSTN3_HUMAN
"MTLLLLPLLLASLLASCSCNKANKHKPWIEAEYQGIVMENDNTVLLNPPLFALDKDAPLRYAGEICGFRLHGSGVPFEAVILDKATGEGLIRAKEPVDCEAQKEHTFTIQAYDCGEGPDGANTKKSHKATVHVRVNDVNEFAPVFVERLYRAAVTEGKLYDRILRVEAIDGDCSPQYSQICYYEILTPNTPFLIDNDGNIENTEKLQYSGERLYKFTVTAYDCGKKRAADDAEVEIQVKPTCKPSWQGWNKRIEYAPGAGSLALFPGIRLETCDEPLWNIQATIELQTSHVAKGCDRDNYSERALRKLCGAATGEVDLLPMPGPNANWTAGLSVHYSQDSSLIYWFNGTQAVQVPLGGPSGLGSGPQDSLSDHFTLSFWMKHGVTPNKGKKEEETIVCNTVQNEDGFSHYSLTVHGCRIAFLYWPLLESARPVKFLWKLEQVCDDEWHHYALNLEFPTVTLYTDGISFDPALIHDNGLIHPPRREPALMIGACWTEEKNKEKEKGDNSTDTTQGDPLSIHHYFHGYLAGFSVRSGRLESREVIECLYACREGLDYRDFESLGKGMKVHVNPSQSLLTLEGDDVETFNHALQHVAYMNTLRFATPGVRPLRLTTAVKCFSEESCVSIPEVEGYVVVLQPDAPQILLSGTAHFARPAVDFEGTNGVPLFPDLQITCSISHQVEAKKDESWQGTVTDTRMSDEIVHNLDGCEISLVGDDLDPERESLLLDTTSLQQRGLELTNTSAYLTIAGVESITVYEEILRQARYRLRHGAALYTRKFRLSCSEMNGRYSSNEFIVEVNVLHSMNRVAHPSHVLSSQQFLHRGHQPPPEMAGHSLASSHRNSMIPSAATLIIVVCVGFLVLMVVLGLVRIHSLHRRVSGAGGPPGASSDPKDPDLFWDDSALTIIVNPMESYQNRQSCVTGAVGGQQEDEDSSDSEVADSPSSDERRIIETPPHRY",
# COR1B_HUMAN
"MSFRKVVRQSKFRHVFGQPVKNDQCYEDIRVSRVTWDSTFCAVNPKFLAVIVEASGGGAFLVLPLSKTGRIDKAYPTVCGHTGPVLDIDWCPHNDEVIASGSEDCTVMVWQIPENGLTSPLTEPVVVLEGHTKRVGIIAWHPTARNVLLSAGCDNVVLIWNVGTAEELYRLDSLHPDLIYNVSWNHNGSLFCSACKDKSVRIIDPRRGTLVAEREKAHEGARPMRAIFLADGKVFTTGFSRMSERQLALWDPENLEEPMALQELDSSNGALLPFYDPDTSVVYVCGKGDSSIRYFEITEEPPYIHFLNTFTSKEPQRGMGSMPKRGLEVSKCEIARFYKLHERKCEPIVMTVPRKSDLFQDDLYPDTAGPEAALEAEEWVSGRDADPILISLREAYVPSKQRDLKISRRNVLSDSRPAMAPGSSHLGAPASTTTAADATPSGSLARAGEAGKLEEVMQELRALRALVKEQGDRICRLEEQLGRMENGDA",
# SYT17_HUMAN
"MAYIQLEPLNEGFLSRISGLLLCRWTCRHCCQKCYESSCCQSSEDEVEILGPFPAQTPPWLMASRSSDKDGDSVHTASEVPLTPRTNSPDGRRSSSDTSKSTYSLTRRISSLESRRPSSPLIDIKPIEFGVLSAKKEPIQPSVLRRTYNPDDYFRKFEPHLYSLDSNSDDVDSLTDEEILSKYQLGMLHFSTQYDLLHNHLTVRVIEARDLPPPISHDGSRQDMAHSNPYVKICLLPDQKNSKQTGVKRKTQKPVFEERYTFEIPFLEAQRRTLLLTVVDFDKFSRHCVIGKVSVPLCEVDLVKGGHWWKALIPSSQNEVELGELLLSLNYLPSAGRLNVDVIRAKQLLQTDVSQGSDPFVKIQLVHGLKLVKTKKTSFLRGTIDPFYNESFSFKVPQEELENASLVFTVFGHNMKSSNDFIGRIVIGQYSSGPSETNHWRRMLNTHRTAVEQWHSLRSRAECDRVSPASLEVT",
# SYT11_HUMAN
"MAEITNIRPSFDVSPVVAGLIGASVLVVCVSVTVFVWSCCHQQAEKKQKNPPYKFIHMLKGISIYPETLSNKKKIIKVRRDKDGPGREGGRRNLLVDAAEAGLLSRDKDPRGPSSGSCIDQLPIKMDYGEELRSPITSLTPGESKTTSPSSPEEDVMLGSLTFSVDYNFPKKALVVTIQEAHGLPVMDDQTQGSDPYIKMTILPDKRHRVKTRVLRKTLDPVFDETFTFYGIPYSQLQDLVLHFLVLSFDRFSRDDVIGEVMVPLAGVDPSTGKVQLTRDIIKRNIQKCISRGELQVSLSYQPVAQRMTVVVLKARHLPKMDITGLSGNPYVKVNVYYGRKRIAKKKTHVKKCTLNPIFNESFIYDIPTDLLPDISIEFLVIDFDRTTKNEVVGRLILGAHSVTASGAEHWREVCESPRKPVAKWHSLSEY",
# LRFN3_HUMAN
"MAILPLLLCLLPLAPASSPPQSATPSPCPRRCRCQTQSLPLSVLCPGAGLLFVPPSLDRRAAELRLADNFIASVRRRDLANMTGLLHLSLSRNTIRHVAAGAFADLRALRALHLDGNRLTSLGEGQLRGLVNLRHLILSNNQLAALAAGALDDCAETLEDLDLSYNNLEQLPWEALGRLGNVNTLGLDHNLLASVPAGAFSRLHKLARLDMTSNRLTTIPPDPLFSRLPLLARPRGSPASALVLAFGGNPLHCNCELVWLRRLAREDDLEACASPPALGGRYFWAVGEEEFVCEPPVVTHRSPPLAVPAGRPAALRCRAVGDPEPRVRWVSPQGRLLGNSSRARAFPNGTLELLVTEPGDGGIFTCIAANAAGEATAAVELTVGPPPPPQLANSTSCDPPRDGDPDALTPPSAASASAKVADTGPPTDRGVQVTEHGATAALVQWPDQRPIPGIRMYQIQYNSSADDILVYRMIPAESRSFLLTDLASGRTYDLCVLAVYEDSATGLTATRPVGCARFSTEPALRPCGAPHAPFLGGTMIIALGGVIVASVLVFIFVLLMRYKVHGGQPPGKAKIPAPVSSVCSQTNGALGPTPTPAPPAPEPAALRAHTVVQLDCEPWGPGHEPVGP",
# AN32E_HUMAN
"MEMKKKINLELRNRSPEEVTELVLDNCLCVNGEIEGLNDTFKELEFLSMANVELSSLARLPSLNKLRKLELSDNIISGGLEVLAEKCPNLTYLNLSGNKIKDLSTVEALQNLKNLKSLDLFNCEITNLEDYRESIFELLQQITYLDGFDQEDNEAPDSEEEDDEDGDEDDEEEEENEAGPPEGYEEEEEEEEEEDEDEDEDEDEAGSELGEGEEEVGLSYLMKEEIQDEEDDDDYVEEGEEEEEEEEGGLRGEKRKRDAEDDGEEEDD",
# P4K2A_HUMAN
"MDETSPLVSPERAQPPDYTFPSGSGAHFPQVPGGAVRVAAAAGSGPSPPGSPGHDRERQPLLDRARGAAAQGQTQTVAAQAQALAAQAAAAAHAAQAHRERNEFPEDPEFEAVVRQAELAIERCIFPERIYQGSSGSYFVKDPQGRIIAVFKPKNEEPYGHLNPKWTKWLQKLCCPCCFGRDCLVLNQGYLSEAGASLVDQKLELNIVPRTKVVYLASETFNYSAIDRVKSRGKRLALEKVPKVGQRFNRIGLPPKVGSFQLFVEGYKDADYWLRRFEAEPLPENTNRQLLLQFERLVVLDYIIRNTDRGNDNWLIKYDCPMDSSSSRDTDWVVVKEPVIKVAAIDNGLAFPLKHPDSWRAYPFYWAWLPQAKVPFSQEIKDLILPKISDPNFVKDLEEDLYELFKKDPGFDRGQFHKQIAVMRGQILNLTQALKDNKSPLHLVQMPPVIVETARSHQRSSSESYTQSFQSRKPFFSWW",
# BEGIN_HUMAN
"MEKLSALQEQKGELRKRLSYTTHKLEKLETEFDSTRHYLEIELRRAQEELEKVTEKLRRIQSNYMALQRINQELEDKLYRMGQHYEEEKRALSHEIVALNSHLLEAKVTIDKLSEDNELYRKDCNLAAQLLQCSQTYGRVHKVSELPSDFQERVSLHMEKHGCSLPSPLCHPAYADSVPTCVIAKVLEKPDPASLSSRLSDASARDLAFCDGVEKPGPRPPYKGDIYCSDTALYCPEERRRDRRPSVDAPVTDVGFLRAQNSTDSAAEEEEEAEAAAFPAGFQHEAFPSYAGSLPTSSSYSSFSATSEEKEHAQASTLTASQQAIYLNSRDELFDRKPPATTYEGSPRFAKATAAVAAPLEAEVAPGFGRTMSPYPAETFRFPASPGPQQALMPPNLWSLRAKPGTARLPGEDMRGQWRPLSVEDIGAYSYPVSAAGRASPCSFSERYYGGAGGSPGKKADGRASPLYASYKADSFSEGDDLSQGHLAEPCFLRAGGDLSLSPGRSADPLPGYAPSEGGDGDRLGVQLCGTASSPEPEQGSRDSLEPSSMEASPEMHPAARLSPQQAFPRTGGSGLSRKDSLTKAQLYGTLLN",
# HNRL1_HUMAN
"MDVRRLKVNELREELQRRGLDTRGLKAELAERLQAALEAEEPDDERELDADDEPGRPGHINEEVETEGGSELEGTAQPPPPGLQPHAEPGGYSGPDGHYAMDNITRQNQFYDTQVIKQENESGYERRPLEMEQQQAYRPEMKTEMKQGAPTSFLPPEASQLKPDRQQFQSRKRPYEENRGRGYFEHREDRRGRSPQPPAEEDEDDFDDTLVAIDTYNCDLHFKVARDRSSGYPLTIEGFAYLWSGARASYGVRRGRVCFEMKINEEISVKHLPSTEPDPHVVRIGWSLDSCSTQLGEEPFSYGYGGTGKKSTNSRFENYGDKFAENDVIGCFADFECGNDVELSFTKNGKWMGIAFRIQKEALGGQALYPHVLVKNCAVEFNFGQRAEPYCSVLPGFTFIQHLPLSERIRGTVGPKSKAECEILMMVGLPAAGKTTWAIKHAASNPSKKYNILGTNAIMDKMRVMGLRRQRNYAGRWDVLIQQATQCLNRLIQIAARKKRNYILDQTNVYGSAQRRKMRPFEGFQRKAIVICPTDEDLKDRTIKRTDEEGKDVPDHAVLEMKANFTLPDVGDFLDEVLFIELQREEADKLVRQYNEEGRKAGPPPEKRFDNRGGGGFRGRGGGGGFQRYENRGPPGGNRGGFQNRGGGSGGGGNYRGGFNRSGGGGYSQNRWGNNNRDNNNSNNRGSYNRAPQQQPPPQQPPPPQPPPQQPPPPPSYSPARNPPGASTYNKNSNIPGSSANTSTPTVSSYSPPQPSYSQPPYNQGGYSQGYTAPPPPPPPPPAYNYGSYGGYNPAPYTPPPPPTAQTYPQPSYNQYQQYAQQWNQYYQNQGQWPPYYGNYDYGSYSGNTQGGTSTQ",
# ABHD6_HUMAN
"MDLDVVNMFVIAGGTLAIPILAFVASFLLWPSALIRIYYWYWRRTLGMQVRYVHHEDYQFCYSFRGRPGHKPSILMLHGFSAHKDMWLSVVKFLPKNLHLVCVDMPGHEGTTRSSLDDLSIDGQVKRIHQFVECLKLNKKPFHLVGTSMGGQVAGVYAAYYPSDVSSLCLVCPAGLQYSTDNQFVQRLKELQGSAAVEKIPLIPSTPEEMSEMLQLCSYVRFKVPQQILQGLVDVRIPHNNFYRKLFLEIVSEKSRYSLHQNMDKIKVPTQIIWGKQDQVLDVSGADMLAKSIANCQVELLENCGHSVVMERPRKTAKLIIDFLASVHNTDNNKKLD",
# SCMC3_HUMAN
"MRGSPGDAERRQRWGRLFEELDSNKDGRVDVHELRQGLARLGGGNPDPGAQQGISSEGDADPDGGLDLEEFSRYLQEREQRLLLMFHSLDRNQDGHIDVSEIQQSFRALGISISLEQAEKILHSMDRDGTMTIDWQEWRDHFLLHSLENVEDVLYFWKHSTVLDIGECLTVPDEFSKQEKLTGMWWKQLVAGAVAGAVSRTGTAPLDRLKVFMQVHASKTNRLNILGGLRSMVLEGGIRSLWRGNGINVLKIAPESAIKFMAYEQIKRAILGQQETLHVQERFVAGSLAGATAQTIIYPMEVLKTRLTLRRTGQYKGLLDCARRILEREGPRAFYRGYLPNVLGIIPYAGIDLAVYETLKNWWLQQYSHDSADPGILVLLACGTISSTCGQIASYPLALVRTRMQAQASIEGGPQLSMLGLLRHILSQEGMRGLYRGIAPNFMKVIPAVSISYVVYENMKQALGVTSR",
# TBB2B_HUMAN
"MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGSYHGDSDLQLERINVYYNEATGNKYVPRAILVDLEPGTMDSVRSGPFGQIFRPDNFVFGQSGAGNNWAKGHYTEGAELVDSVLDVVRKESESCDCLQGFQLTHSLGGGTGSGMGTLLISKIREEYPDRIMNTFSVMPSPKVSDTVVEPYNATLSVHQLVENTDETYCIDNEALYDICFRTLKLTTPTYGDLNHLVSATMSGVTTCLRFPGQLNADLRKLAVNMVPFPRLHFFMPGFAPLTSRGSQQYRALTVPELTQQMFDSKNMMAACDPRHGRYLTVAAIFRGRMSMKEVDEQMLNVQNKNSSYFVEWIPNNVKTAVCDIPPRGLKMSATFIGNSTAIQELFKRISEQFTAMFRRKAFLHWYTGEGMDEMEFTEAESNMNDLVSEYQQYQDATADEQGEFEEEEGEDEA",
# TMUB1_HUMAN
"MTLIEGVGDEVTVLFSVLACLLVLALAWVSTHTAEGGDPLPQPSGTPTPSQPSAAMAATDSMRGEAPGAETPSLRHRGQAAQPEPSTGFTATPPAPDSPQEPLVLRLKFLNDSEQVARAWPHDTIGSLKRTQFPGREQQVRLIYQGQLLGDDTQTLGSLHLPPNCVLHCHVSTRVGPPNPPCPPGSEPGPSGLEIGSLLLPLLLLLLLLLWYCQIQYRPFFPLTATLGLAGFTLLLSLLAFAMYRP",
# RFIP5_HUMAN
"MALVRGAEPAAGPSRWLPTHVQVTVLRARGLRGKSSGAGSTSDAYTVIQVGREKYSTSVVEKTHGCPEWREECSFELPPGALDGLLRAQEADAGPAPWAASSAAACELVLTTMHRSLIGVDKFLGQATVALDEVFGAGRAQHTQWYKLHSKPGKKEKERGEIEVTIQFTRNNLSASMFDLSMKDKPRSPFSKIRDKMKGKKKYDLESASAILPSSAIEDPDLGSLGKMGKAKGFFLRNKLRKSSLTQSNTSLGSDSTLSSASGSLAYQGPGAELLTRSPSRSSWLSTEGGRDSAQSPKLFTHKRTYSDEANQMRVAPPRALLDLQGHLDAASRSSLCVNGSHIYNEEPQGPVRHRSSISGSLPSSGSLQAVSSRFSEEGPRSTDDTWPRGSRSNSSSEAVLGQEELSAQAKVLAPGASHPGEEEGARLPEGKPVQVATPIVASSEAVAEKEGARKEERKPRMGLFHHHHQGLSRSELGRRSSLGEKGGPILGASPHHSSSGEEKAKSSWFGLREAKDPTQKPSPHPVKPLSAAPVEGSPDRKQSRSSLSIALSSGLEKLKTVTSGSIQPVTQAPQAGQMVDTKRLKDSAVLDQSAKYYHLTHDELISLLLQRERELSQRDEHVQELESYIDRLLVRIMETSPTLLQIPPGPPK",
# AP1M1_HUMAN
"MSASAVYVLDLKGKVLICRNYRGDVDMSEVEHFMPILMEKEEEGMLSPILAHGGVRFMWIKHNNLYLVATSKKNACVSLVFSFLYKVVQVFSEYFKELEEESIRDNFVIIYELLDELMDFGYPQTTDSKILQEYITQEGHKLETGAPRPPATVTNAVSWRSEGIKYRKNEVFLDVIESVNLLVSANGNVLRSEIVGSIKMRVFLSGMPELRLGLNDKVLFDNTGRGKSKSVELEDVKFHQCVRLSRFENDRTISFIPPDGEFELMSYRLNTHVKPLIWIESVIEKHSHSRIEYMIKAKSQFKRRSTANNVEIHIPVPNDADSPKFKTTVGSVKWVPENSEIVWSIKSFPGGKEYLMRAHFGLPSVEAEDKEGKPPISVKFEIPYFTTSGIQVRYLKIIEKSGYQALPWVRYITQNGDYQLRTQ",
# TDRD1_HUMAN
"MSVKSPFNVMSRNNLEAPPCKMTEPFNFEKNENKLPPHESLRSPGTLPNHPNFRLKSSENGNKKNNFLLCEQTKQYLASQEDNSVSSNPNGINGEVVGSKGDRKKLPAGNSVSPPSAESNSPPKEVNIKPGNNVRPAKSKKLNKLVENSLSISNPGLFTSLGPPLRSTTCHRCGLFGSLRCSQCKQTYYCSTACQRRDWSAHSIVCRPVQPNFHKLENKSSIETKDVEVNNKSDCPLGVTKEIAIWAERIMFSDLRSLQLKKTMEIKGTVTEFKHPGDFYVQLYSSEVLEYMNQLSASLKETYANVHEKDYIPVKGEVCIAKYTVDQTWNRAIIQNVDVQQKKAHVLYIDYGNEEIIPLNRIYHLNRNIDLFPPCAIKCFVANVIPAEGNWSSDCIKATKPLLMEQYCSIKIVDILEEEVVTFAVEVELPNSGKLLDHVLIEMGYGLKPSGQDSKKENADQSDPEDVGKMTTENNIVVDKSDLIPKVLTLNVGDEFCGVVAHIQTPEDFFCQQLQSGRKLAELQASLSKYCDQLPPRSDFYPAIGDICCAQFSEDDQWYRASVLAYASEESVLVGYVDYGNFEILSLMRLCPIIPKLLELPMQAIKCVLAGVKPSLGIWTPEAICLMKKLVQNKIITVKVVDKLENSSLVELIDKSETPHVSVSKVLLDAGFAVGEQSMVTDKPSDVKETSVPLGVEGKVNPLEWTWVELGVDQTVDVVVCVIYSPGEFYCHVLKEDALKKLNDLNKSLAEHCQQKLPNGFKAEIGQPCCAFFAGDGSWYRALVKEILPNGHVKVHFVDYGNIEEVTADELRMISSTFLNLPFQGIRCQLADIQSRNKHWSEEAITRFQMCVAGIKLQARVVEVTENGIGVELTDLSTCYPRIISDVLIDEHLVLKSASPHKDLPNDRLVNKHELQVHVQGLQATSSAEQWKTIELPVDKTIQANVLEIISPNLFYALPKGMPENQEKLCMLTAELLEYCNAPKSRPPYRPRIGDACCAKYTSDDFWYRAVVLGTSDTDVEVLYADYGNIETLPLCRVQPITSSHLALPFQIIRCSLEGLMELNGSSSQLIIMLLKNFMLNQNVMLSVKGITKNVHTVSVEKCSENGTVDVADKLVTFGLAKNITPQRQSALNTEKMYRMNCCCTELQKQVEKHEHILLFLLNNSTNQNKFIEMKKLLKS",
# PACN1_HUMAN
"MSSSYDEASLAPEETTDSFWEVGNYKRTVKRIDDGHRLCNDLMNCVQERAKIEKAYGQQLTDWAKRWRQLIEKGPQYGSLERAWGAIMTEADKVSELHQEVKNNLLNEDLEKVKNWQKDAYHKQIMGGFKETKEAEDGFRKAQKPWAKKMKELEAAKKAYHLACKEEKLAMTREMNSKTEQSVTPEQQKKLQDKVDKCKQDVQKTQEKYEKVLEDVGKTTPQYMENMEQVFEQCQQFEEKRLVFLKEVLLDIKRHLNLAENSSYIHVYRELEQAIRGADAQEDLRWFRSTSGPGMPMNWPQFEEWNPDLPHTTTKKEKQPKKAEGVALTNATGAVESTSQAGDRGSVSSYDRGQPYATEWSDDESGNPFGGSETNGGANPFEDDSKGVRVRALYDYDGQEQDELSFKAGDELTKLGEEDEQGWCRGRLDSGQLGLYPANYVEAI",
# CADM1_HUMAN
"MASVVLPSGSQCAAAAAAAAPPGLRLRLLLLLFSAAALIPTGDGQNLFTKDVTVIEGEVATISCQVNKSDDSVIQLLNPNRQTIYFRDFRPLKDSRFQLLNFSSSELKVSLTNVSISDEGRYFCQLYTDPPQESYTTITVLVPPRNLMIDIQKDTAVEGEEIEVNCTAMASKPATTIRWFKGNTELKGKSEVEEWSDMYTVTSQLMLKVHKEDDGVPVICQVEHPAVTGNLQTQRYLEVQYKPQVHIQMTYPLQGLTREGDALELTCEAIGKPQPVMVTWVRVDDEMPQHAVLSGPNLFINNLNKTDNGTYRCEASNIVGKAHSDYMLYVYDPPTTIPPPTTTTTTTTTTTTTILTIITDSRAGEEGSIRAVDHAVIGGVVAVVVFAMLCLLIILGRYFARHKGTYFTHEAKGADDAADADTAIINAEGGQNNSEEKKEYFI",
# SHAN3_HUMAN
"MQLSRAAAAAAAAPAEPPEPLSPAPAPAPAPPGPLPRSAADGAPAGGKGGPGRRAESPGAPFPGASGPGPGPGAGMDGPGASAVVVRVGIPDLQQTKCLRLDPAAPVWAAKQRVLCALNHSLQDALNYGLFQPPSRGRAGKFLDEERLLQEYPPNLDTPLPYLEFRYKRRVYAQNLIDDKQFAKLHTKANLKKFMDYVQLHSTDKVARLLDKGLDPNFHDPDSGECPLSLAAQLDNATDLLKVLKNGGAHLDFRTRDGLTAVHCATRQRNAAALTTLLDLGASPDYKDSRGLTPLYHSALGGGDALCCELLLHDHAQLGITDENGWQEIHQACRFGHVQHLEHLLFYGADMGAQNASGNTALHICALYNQESCARVLLFRGANRDVRNYNSQTAFQVAIIAGNFELAEVIKTHKDSDVVPFRETPSYAKRRRLAGPSGLASPRPLQRSASDINLKGEAQPAASPGPSLRSLPHQLLLQRLQEEKDRDRDADQESNISGPLAGRAGQSKISPSGPGGPGPAPGPGPAPPAPPAPPPRGPKRKLYSAVPGRKFIAVKAHSPQGEGEIPLHRGEAVKVLSIGEGGFWEGTVKGRTGWFPADCVEEVQMRQHDTRPETREDRTKRLFRHYTVGSYDSLTSHSDYVIDDKVAVLQKRDHEGFGFVLRGAKAETPIEEFTPTPAFPALQYLESVDVEGVAWRAGLRTGDFLIEVNGVNVVKVGHKQVVALIRQGGNRLVMKVVSVTRKPEEDGARRRAPPPPKRAPSTTLTLRSKSMTAELEELASIRRRKGEKLDEMLAAAAEPTLRPDIADADSRAATVKQRPTSRRITPAEISSLFERQGLPGPEKLPGSLRKGIPRTKSVGEDEKLASLLEGRFPRSTSMQDPVREGRGIPPPPQTAPPPPPAPYYFDSGPPPAFSPPPPPGRAYDTVRSSFKPGLEARLGAGAAGLYEPGAALGPLPYPERQKRARSMIILQDSAPESGDAPRPPPAATPPERPKRRPRPPGPDSPYANLGAFSASLFAPSKPQRRKSPLVKQLQVEDAQERAALAVGSPGPGGGSFAREPSPTHRGPRPGGLDYGAGDGPGLAFGGPGPAKDRRLEERRRSTVFLSVGAIEGSAPGADLPSLQPSRSIDERLLGTGPTAGRDLLLPSPVSALKPLVSGPSLGPSGSTFIHPLTGKPLDPSSPLALALAARERALASQAPSRSPTPVHSPDADRPGPLFVDVQARDPERGSLASPAFSPRSPAWIPVPARREAEKVPREERKSPEDKKSMILSVLDTSLQRPAGLIVVHATSNGQEPSRLGGAEEERPGTPELAPAPMQSAAVAEPLPSPRAQPPGGTPADAGPGQGSSEEEPELVFAVNLPPAQLSSSDEETREELARIGLVPPPEEFANGVLLATPLAGPGPSPTTVPSPASGKPSSEPPPAPESAADSGVEEADTRSSSDPHLETTSTISTVSSMSTLSSESGELTDTHTSFADGHTFLLEKPPVPPKPKLKSPLGKGPVTFRDPLLKQSSDSELMAQQHHAASAGLASAAGPARPRYLFQRRSKLWGDPVESRGLPGPEDDKPTVISELSSRLQQLNKDTRSLGEEPVGGLGSLLDPAKKSPIAAARLFSSLGELSSISAQRSPGGPGGGASYSVRPSGRYPVARRAPSPVKPASLERVEGLGAGAGGAGRPFGLTPPTILKSSSLSIPHEPKEVRFVVRSVSARSRSPSPSPLPSPASGPGPGAPGPRRPFQQKPLQLWSKFDVGDWLESIHLGEHRDRFEDHEIEGAHLPALTKDDFVELGVTRVGHRMNIERALRQLDGS",
# SE6L1_HUMAN
"MPAARPPAAGLRGISLFLALLLGSPAAALERDALPEGDASPLGPYLLPSGAPERGSPGKEHPEERVVTAPPSSSQSAEVLGELVLDGTAPSAHHDIPALSPLLPEEARPKHALPPKKKLPSLKQVNSARKQLRPKATSAATVQRAGSQPASQGLDLLSSSTEKPGPPGDPDPIVASEEASEVPLWLDRKESAVPTTPAPLQISPFTSQPYVAHTLPQRPEPGEPGPDMAQEAPQEDTSPMALMDKGENELTGSASEESQETTTSTIITTTVITTEQAPALCSVSFSNPEGYIDSSDYPLLPLNNFLECTYNVTVYTGYGVELQVKSVNLSDGELLSIRGVDGPTLTVLANQTLLVEGQVIRSPTNTISVYFRTFQDDGLGTFQLHYQAFMLSCNFPRRPDSGDVTVMDLHSGGVAHFHCHLGYELQGAKMLTCINASKPHWSSQEPICSAPCGGAVHNATIGRVLSPSYPENTNGSQFCIWTIEAPEGQKLHLHFERLLLHDKDRMTVHSGQTNKSALLYDSLQTESVPFEGLLSEGNTIRIEFTSDQARAASTFNIRFEAFEKGHCYEPYIQNGNFTTSDPTYNIGTIVEFTCDPGHSLEQGPAIIECINVRDPYWNDTEPLCRAMCGGELSAVAGVVLSPNWPEPYVEGEDCIWKIHVGEEKRIFLDIQFLNLSNSDILTIYDGDEVMPHILGQYLGNSGPQKLYSSTPDLTIQFHSDPAGLIFGKGQGFIMNYIEVSRNDSCSDLPEIQNGWKTTSHTELVRGARITYQCDPGYDIVGSDTLTCQWDLSWSSDPPFCEKIMYCTDPGEVDHSTRLISDPVLLVGTTIQYTCNPGFVLEGSSLLTCYSRETGTPIWTSRLPHCVSEESLACDNPGLPENGYQILYKRLYLPGESLTFMCYEGFELMGEVTIRCILGQPSHWNGPLPVCKVNQDSFEHALEVAEAAAETSLEGGNMALAIFIPVLIISLLLGGAYIYITRCRYYSNLRLPLMYSHPYSQITVETEFDNPIYETGETREYEVSI",
# TBD2A_HUMAN
"MEGAGENAPESSSSAPGSEESARDPQVPPPEEESGDCARSLEAVPKKLCGYLSKFGGKGPIRGWKSRWFFYDERKCQLYYSRTAQDANPLDSIDLSSAVFDCKADAEEGIFEIKTPSRVITLKAATKQAMLYWLQQLQMKRWEFHNSPPAPPATPDAALAGNGPVLHLELGQEEAELEEFLCPVKTPPGLVGVAAALQPFPALQNISLKHLGTEIQNTMHNIRGNKQAQGTGHEPPGEDSPQSGEPQREEQPLASDASTPGREPEDSPKPAPKPSLTISFAQKAKRQNNTFPFFSEGITRNRTAQEKVAALEQQVLMLTKELKSQKELVKILHKALEAAQQEKRASSAYLAAAEDKDRLELVRHKVRQIAELGRRVEALEQERESLAHTASLREQQVQELQQHVQLLMDKNHAKQQVICKLSEKVTQDFTHPPDQSPLRPDAANRDFLSQQGKIEHLKDDMEAYRTQNCFLNSEIHQVTKIWRKVAEKEKALLTKCAYLQARNCQVESKYLAGLRRLQEALGDEASECSELLRQLVQEALQWEAGEASSDSIELSPISKYDEYGFLTVPDYEVEDLKLLAKIQALESRSHHLLGLEAVDRPLRERWAALGDLVPSAELKQLLRAGVPREHRPRVWRWLVHLRVQHLHTPGCYQELLSRGQAREHPAARQIELDLNRTFPNNKHFTCPTSSFPDKLRRVLLAFSWQNPTIGYCQGLNRLAAIALLVLEEEESAFWCLVAIVETIMPADYYCNTLTASQVDQRVLQDLLSEKLPRLMAHLGQHHVDLSLVTFNWFLVVFADSLISNILLRVWDAFLYEGTKVVFRYALAIFKYNEKEILRLQNGLEIYQYLRFFTKTISNSRKLMNIAFNDMNPFRMKQLRQLRMVHRERLEAELRELEQLKAEYLERRASRRRAVSEGCASEDEVEGEA",
# CPEB1_HUMAN
"MALSLEEEAGRIKDCWDNQEAPALSTCSNANIFRRINAILDNSLDFSRVCTTPINRGIHDHLPDFQDSEETVTSRMLFPTSAQESSRGLPDANDLCLGLQSLSLTGWDRPWSTQDSDSSAQSSTHSVLSMLHNPLGNVLGKPPLSFLPLDPLGSDLVDKFPAPSVRGSRLDTRPILDSRSSSPSDSDTSGFSSGSDHLSDLISSLRISPPLPFLSLSGGGPRDPLKMGVGSRMDQEQAALAAVTPSPTSASKRWPGASVWPSWDLLEAPKDPFSIEREARLHRQAAAVNEATCTWSGQLPPRNYKNPIYSCKVFLGGVPWDITEAGLVNTFRVFGSLSVEWPGKDGKHPRCPPKGNMPKGYVYLVFELEKSVRSLLQACSHDPLSPDGLSEYYFKMSSRRMRCKEVQVIPWVLADSNFVRSPSQRLDPSRTVFVGALHGMLNAEALAAILNDLFGGVVYAGIDTDKHKYPIGSGRVTFNNQRSYLKAVSAAFVEIKTTKFTKKVQIDPYLEDSLCHICSSQPGPFFCRDQVCFKYFCRSCWHWRHSMEGLRHHSPLMRNQKNRDSS",
# PPAT_HUMAN
"MAGLGFWGHPAGPLLLLLLLVLPPRALPEGPLVFVALVFRHGDRAPLASYPMDPHKEVASTLWPRGLGQLTTEGVRQQLELGRFLRSRYEAFLSPEYRREEVYIRSTDFDRTLESAQANLAGLFPEAAPGSPEARWRPIPVHTVPVAEDKLLRFPMRSCPRYHELLREATEAAEYQEALEGWTGFLSRLENFTGLSLVGEPLRRAWKVLDTLMCQQAHGLPLPAWASPDVLRTLAQISALDIGAHVGPPRAAEKAQLTGGILLNAILANFSRVQRLGLPLKMVMYSAHDSTLLALQGALGLYDGHTPPYAACLGFEFRKHLGNPAKDGGNVTVSLFYRNDSAHLPLPLSLPGCPAPCPLGRFYQLTAPARPPAHGVSCHGPYEAAIPPAPVVPLLAGAVAVLVALSLGLGLLAWRPGCLRALGGPV",
# RTN4R_HUMAN
"MKRASAGGSRLLAWVLWLQAWQVAAPCPGACVCYNEPKVTTSCPQQGLQAVPVGIPAASQRIFLHGNRISHVPAASFRACRNLTILWLHSNVLARIDAAAFTGLALLEQLDLSDNAQLRSVDPATFHGLGRLHTLHLDRCGLQELGPGLFRGLAALQYLYLQDNALQALPDDTFRDLGNLTHLFLHGNRISSVPERAFRGLHSLDRLLLHQNRVAHVHPHAFRDLGRLMTLYLFANNLSALPTEALAPLRALQYLRLNDNPWVCDCRARPLWAWLQKFRGSSSEVPCSLPQRLAGRDLKRLAANDLQGCAVATGPYHPIWTGRATDEEPLGLPKCCQPDAADKASVLEPGRPASAGNALKGRVPPGDSPPGNGSGPRHINDSPFGTLPGSAEPPLTAVRPEGSEPPGFPTSGPRRRPGCSRKNRTRSHCRLGQAGSGGGGTGDSEGSGALPSLTCSLTPLGLALVLWTVLGPC",
# TRIM9_HUMAN
"MEEMEEELKCPVCGSFYREPIILPCSHNLCQACARNILVQTPESESPQSHRAAGSGVSDYDYLDLDKMSLYSEADSGYGSYGGFASAPTTPCQKSPNGVRVFPPAMPPPATHLSPALAPVPRNSCITCPQCHRSLILDDRGLRGFPKNRVLEGVIDRYQQSKAAALKCQLCEKAPKEATVMCEQCDVFYCDPCRLRCHPPRGPLAKHRLVPPAQGRVSRRLSPRKVSTCTDHELENHSMYCVQCKMPVCYQCLEEGKHSSHEVKALGAMWKLHKSQLSQALNGLSDRAKEAKEFLVQLRNMVQQIQENSVEFEACLVAQCDALIDALNRRKAQLLARVNKEHEHKLKVVRDQISHCTVKLRQTTGLMEYCLEVIKENDPSGFLQISDALIRRVHLTEDQWGKGTLTPRMTTDFDLSLDNSPLLQSIHQLDFVQVKASSPVPATPILQLEECCTHNNSATLSWKQPPLSTVPADGYILELDDGNGGQFREVYVGKETMCTVDGLHFNSTYNARVKAFNKTGVSPYSKTLVLQTSEVAWFAFDPGSAHSDIILSNDNLTVTCSSYDDRVVLGKTGFSKGIHYWELTVDRYDNHPDPAFGVARMDVMKDVMLGKDDKAWAMYVDNNRSWFMHNNSHTNRTEGGITKGATIGVLLDLNRKNLTFFINDEQQGPIAFDNVEGLFFPAVSLNRNVQVTLHTGLPVPDFYSSRASIA",
# CNTP4_HUMAN
"MGSVTGAVLKTLLLLSTQNWNRVEAGNSYDCDDPLVSALPQASFSSSSELSSSHGPGFARLNRRDGAGGWSPLVSNKYQWLQIDLGERMEVTAVATQGGYGSSNWVTSYLLMFSDSGWNWKQYRQEDSIWGFSGNANADSVVYYRLQPSIKARFLRFIPLEWNPKGRIGMRIEVFGCAYRSEVVDLDGKSSLLYRFDQKSLSPIKDIISLKFKTMQSDGILLHREGPNGDHITLQLRRARLFLLINSGEAKLPSTSTLVNLTLGSLLDDQHWHSVLIQRLGKQVNFTVDEHRHHFHARGEFNLMNLDYEISFGGIPAPGKSVSFPHRNFHGCLENLYYNGVDIIDLAKQQKPQIIAMGNVSFSCSQPQSMPVTFLSSRSYLALPDFSGEEEVSATFQFRTWNKAGLLLFSELQLISGGILLFLSDGKLKSNLYQPGKLPSDITAGVELNDGQWHSVSLSAKKNHLSVAVDGQMASAAPLLGPEQIYSGGTYYFGGCPDKSFGSKCKSPLGGFQGCMRLISISGKVVDLISVQQGSLGNFSDLQIDSCGISDRCLPNYCEHGGECSQSWSTFHCNCTNTGYRGATCHNSIYEQSCEAYKHRGNTSGFYYIDSDGSGPLEPFLLYCNMTETAWTIIQHNGSDLTRVRNTNPENPYAGFFEYVASMEQLQATINRAEHCEQEFTYYCKKSRLVNKQDGTPLSWWVGRTNETQTYWGGSSPDLQKCTCGLEGNCIDSQYYCNCDADRNEWTNDTGLLAYKEHLPVTKIVITDTGRLHSEAAYKLGPLLCQGDRSFWNSASFDTEASYLHFPTFHGELSADVSFFFKTTASSGVFLENLGIADFIRIELRSPTVVTFSFDVGNGPFEISVQSPTHFNDNQWHHVRVERNMKEASLQVDQLTPKTQPAPADGHVLLQLNSQLFVGGTATRQRGFLGCIRSLQLNGMTLDLEERAQVTPEVQPGCRGHCSSYGKLCRNGGKCRERPIGFFCDCTFSAYTGPFCSNEISAYFGSGSSVIYNFQENYLLSKNSSSHAASFHGDMKLSREMIKFSFRTTRTPSLLLFVSSFYKEYLSVIIAKNGSLQIRYKLNKYQEPDVVNFDFKNMADGQLHHIMINREEGVVFIEIDDNRRRQVHLSSGTEFSAVKSLVLGRILEHSDVDQDTALAGAQGFTGCLSAVQLSHVAPLKAALHPSHPDPVTVTGHVTESSCMAQPGTDATSRERTHSFADHSGTIDDREPLANAIKSDSAVIGGLIAVVIFILLCITAIAVRIYQQKRLYKRSEAKRSENVDSAEAVLKSELNIQNAVNENQKEYFF",
# ZDHC5_HUMAN
"MPAESGKRFKPSKYVPVSAAAIFLVGATTLFFAFTCPGLSLYVSPAVPIYNAIMFLFVLANFSMATFMDPGIFPRAEEDEDKEDDFRAPLYKTVEIKGIQVRMKWCATCRFYRPPRCSHCSVCDNCVEEFDHHCPWVNNCIGRRNYRYFFLFLLSLTAHIMGVFGFGLLYVLYHIEELSGVRTAVTMAVMCVAGLFFIPVAGLTGFHVVLVARGRTTNEQVTGKFRGGVNPFTNGCCNNVSRVLCSSPAPRYLGRPKKEKTIVIRPPFLRPEVSDGQITVKIMDNGIQGELRRTKSKGSLEITESQSADAEPPPPPKPDLSRYTGLRTHLGLATNEDSSLLAKDSPPTPTMYKYRPGYSSSSTSAAMPHSSSAKLSRGDSLKEPTSIAESSRHPSYRSEPSLEPESFRSPTFGKSFHFDPLSSGSRSSSLKSAQGTGFELGQLQSIRSEGTTSTSYKSLANQTRNGSLSYDSLLTPSDSPDFESVQAGPEPDPPLGYTSPFLSARLAQQREAERHPRLVPTGPTHREPSPVRYDNLSRHIVASLQEREKLLRQSPPLPGREEEPGLGDSGIQSTPGSGHAPRTSSSSDDSKRSPLGKTPLGRPAVPRFGKPDGLRGRGVGSPEPGPTAPYLGRSMSYSSQKAQPGVSETEEVALQPLLTPKDEVQLKTTYSKSNGQPKSLGSASPGPGQPPLSSPTRGGVKKVSGVGGTTYEISV",
# SEM4C_HUMAN
"MAPHWAVWLLAARLWGLGIGAEVWWNLVPRKTVSSGELATVVRRFSQTGIQDFLTLTLTEPTGLLYVGAREALFAFSMEALELQGAISWEAPVEKKTECIQKGKNNQTECFNFIRFLQPYNASHLYVCGTYAFQPKCTYVNMLTFTLEHGEFEDGKGKCPYDPAKGHAGLLVDGELYSATLNNFLGTEPIILRNMGPHHSMKTEYLAFWLNEPHFVGSAYVPESVGSFTGDDDKVYFFFRERAVESDCYAEQVVARVARVCKGDMGGARTLQRKWTTFLKARLACSAPNWQLYFNQLQAMHTLQDTSWHNTTFFGVFQAQWGDMYLSAICEYQLEEIQRVFEGPYKEYHEEAQKWDRYTDPVPSPRPGSCINNWHRRHGYTSSLELPDNILNFVKKHPLMEEQVGPRWSRPLLVKKGTNFTHLVADRVTGLDGATYTVLFIGTGDGWLLKAVSLGPWVHLIEELQLFDQEPMRSLVLSQSKKLLFAGSRSQLVQLPVADCMKYRSCADCVLARDPYCAWSVNTSRCVAVGGHSGSLLIQHVMTSDTSGICNLRGSKKVRPTPKNITVVAGTDLVLPCHLSSNLAHARWTFGGRDLPAEQPGSFLYDARLQALVVMAAQPRHAGAYHCFSEEQGARLAAEGYLVAVVAGPSVTLEARAPLENLGLVWLAVVALGAVCLVLLLLVLSLRRRLREELEKGAKATERTLVYPLELPKEPTSPPFRPCPEPDEKLWDPVGYYYSDGSLKIVPGHARCQPGGGPPSPPPGIPGQPLPSPTRLHLGGGRNSNANGYVRLQLGGEDRGGLGHPLPELADELRRKLQQRQPLPDSNPEESSV",
# PHAR1_HUMAN
"MDYPKMDYFLDVESAHRLLDVESAQRFFYSQGAQARRATLLLPPTLMAASSEDDIDRRPIRRVRSKSDTPYLAEARISFNLGAAEEVERLAAMRSDSLVPGTHTPPIRRRSKFANLGRIFKPWKWRKKKSEKFKHTSAALERKISMRQSREELIKRGVLKEIYDKDGELSISNEEDSLENGQSLSSSQLSLPALSEMEPVPMPRDPCSYEVLQPSDIMDGPDPGAPVKLPCLPVKLSPPLPPKKVMICMPVGGPDLSLVSYTAQKSGQQGVAQHHHTVLPSQIQHQLQYGSHGQHLPSTTGSLPMHPSGCRMIDELNKTLAMTMQRLESSEQRVPCSTSYHSSGLHSGDGVTKAGPMGLPEIRQVPTVVIECDDNKENVPHESDYEDSSCLYTREEEEEEEDEDDDSSLYTSSLAMKVCRKDSLAIKLSNRPSKRELEEKNILPRQTDEERLELRQQIGTKLTRRLSQRPTAEELEQRNILKPRNEQEEQEEKREIKRRLTRKLSQRPTVEELRERKILIRFSDYVEVADAQDYDRRADKPWTRLTAADKAAIRKELNEFKSTEMEVHELSRHLTRFHRP",
# TANC1_HUMAN
"MLKAVLKKSREGGKGGKKEAGSDFGPETSPVLHLDHSADSPVSSLPTAEDTYRVSLAKGVSMSLPSSPLLPRQSHLVQSRVNKKSPGPVRKPKYVESPRVPGDAVIMPFREVAKPTEPDEHEAKADNEPSCSPAAQELLTRLGFLLGEGIPSATHITIEDKNETMCTALSQGISPCSTLTSSTASPSTDSPCSTLNSCVSKTAANKSPCETISSPSSTLESKDSGIIATITSSSENDDRSGSSLEWNKDGNLRLGVQKGVLHDRRADNCSPVAEEETTGSAESTLPKAESSAGDGPVPYSQGSSSLIMPRPNSVAATSSTKLEDLSYLDGQRNAPLRTSIRLPWHNTAGGRAQEVKARFAPYKPQDILLKPLLFEVPSITTDSVFVGRDWLFHQIEENLRNTELAENRGAVVVGNVGFGKTAIISKLVALSCHGSRMRQIASNSPGSSPKTSDPTQDLHFTPLLSPSSSTSASSTAKTPLGSISAENQRPREDAVKYLASKVVAYHYCQADNTYTCLVPEFVHSIAALLCRSHQLAAYRDLLIKEPQLQSMLSLRSCVQDPVAAFKRGVLEPLTNLRNEQKIPEEEYIILIDGLNEAEFHKPDYGDTLSSFITKIISKFPAWLKLIVTVRANFQEIISALPFVKLSLDDFPDNKDIHSDLHAYVQHRVHSSQDILSNISLNGKADATLIGKVSSHLVLRSLGSYLYLKLTLDLFQRGHLVIKSASYKVVPVSLSELYLLQCNMKFMTQSAFERALPILNVALASLHPMTDEQIFQAINAGHIQGEQGWEDFQQRMDALSCFLIKRRDKTRMFCHPSFREWLVWRADGENTAFLCEPRNGHALLAFMFSRQEGKLNRQQTMELGHHILKAHIFKGLSKKTGISSSHLQALWIGYSTEGLSAALASLRNLYTPNVKVSRLLILGGANVNYRTEVLNNAPILCVQSHLGHEEVVTLLLEFGACLDGTSENGMTALCYAAAAGHMKLVCLLTKKGVRVDHLDKKGQCALVHSALRGHGDILQYLLTCEWSPGPPQPGTLRKSHALQQALTAAASMGHSSVVQCLLGMEKEHEVEVNGTDTLWGETALTAAAGRGKLEVCELLLGHGAAVSRTNRRGVPPLFCAARQGHWQIVRLLLERGCDVNLSDKQGRTPLMVAACEGHLSTVEFLLSKGAALSSLDKEGLSALSWACLKGHRAVVQYLVEEGAAIDQTDKNGRTPLDLAAFYGDAETVLYLVEKGAVIEHVDHSGMRPLDRAIGCRNTSVVVALLRKGAKLGNAAWAMATSKPDILIILLQKLMEEGNVMYKKGKMKEAAQRYQYALRKFPREGFGEDMRPFNELRVSLYLNLSRCRRKTNDFGMAEEFASKALELKPKSYEAFYARARAKRNSRQFVAALADLQEAVKLCPTNQEVKRLLARVEEECKQLQRSQQQKQQGPLPAPLNDSENEEDTPTPGLSDHFHSEETEEEETSPQEESVSPTPRSQPSSSVPSSYIRNLQEGLQSKGRPVSPQSRAGIGKSLREPVAQPGLLLQPSKQAQIVKTSQHLGSGQSAVRNGSMKVQISSQNPPPSPMPGRIAATPAGSRTQHLEGTGTFTTRAGCGHFGDRLGPSQNVRLQCGENGPAHPLPSKTKTTERLLSHSSVAVDAAPPNQGGLATCSDVRHPASLTSSGSSGSPSSSIKMSSSTSSLTSSSSFSDGFKVQGPDTRIKDKVVTHVQSGTAEHRPRNTPFMGIMDKTARFQQQSNPPSRSWHCPAPEGLLTNTSSAAGLQSANTEKPSLMQVGGYNNQAKTCSVSTLSASVHNGAQVKELEESKCQIPVHSQENRITKTVSHLYQESISKQQPHISNEAHRSHLTAAKPKRSFIESNV",
# GRIP2_HUMAN
"MLCGLSRETPGEADDGPYSKGGKDAGGADVSLACRRQSIPEEFRGITVVELIKKEGSTLGLTISGGTDKDGKPRVSNLRPGGLAARSDLLNIGDYIRSVNGIHLTRLRHDEIITLLKNVGERVVLEVEYELPPPAPENNPRIISKTVDVSLYKEGNSFGFVLRGGAHEDGHKSRPLVLTYVRPGGPADREGSLKVGDRLLSVDGIPLHGASHATALATLRQCSHEALFQVEYDVATPDTVANASGPLMVEIVKTPGSALGISLTTTSLRNKSVITIDRIKPASVVDRSGALHPGDHILSIDGTSMEHCSLLEATKLLASISEKVRLEILPVPQSQRPLRPSEAVKVQRSEQLHRWDPCVPSCHSPRPGHCRMPTWATPAGQDQSRSLSSTPFSSPTLNHAFSCNNPSTLPRGSQPMSPRTTMGRRRQRRREHKSSLSLASSTVGPGGQIVHTETTEVVLCGDPLSGFGLQLQGGIFATETLSSPPLVCFIEPDSPAERCGLLQVGDRVLSINGIATEDGTMEEANQLLRDAALAHKVVLEVEFDVAESVIPSSGTFHVKLPKKRSVELGITISSASRKRGEPLIISDIKKGSVAHRTGTLEPGDKLLAIDNIRLDNCPMEDAVQILRQCEDLVKLKIRKDEDNSDELETTGAVSYTVELKRYGGPLGITISGTEEPFDPIVISGLTKRGLAERTGAIHVGDRILAINNVSLKGRPLSEAIHLLQVAGETVTLKIKKQLDRPLLPRKSGSLSETSDADEDPADALKGGLPAARFSPAVPSVDSAVESWDSSATEGGFGGPGSYTPQAAARGTTPQERRPGWLRGSPPPTEPRRTSYTPTPADESFPEEEEEDDWEPPTSPAPGPAREEGFWRMFGEALEDLESCGQSELLRELEASIMTGTVQRVALEGRPGHRPWQRGREVRASPAEMEELLLPTPLEMHKVTLHKDPMRHDFGFSVSDGLLEKGVYVHTVRPDGPAHRGGLQPFDRVLQVNHVRTRDFDCCLAVPLLAEAGDVLELIISRKPHTAHSSRAPRSPGPSSPRML",
# RHG39_HUMAN
"MSQTQDYECRSHNVDLPESRIPGSNTRLEWVEIIEPRTRERMYANLVTGECVWDPPAGVRIKRTSENQWWELFDPNTSRFYYYNASTQRTVWHRPQGCDIIPLAKLQTLKQNTESPRASAESSPGRGSSVSREGSTSSSLEPEPDTEKAQELPARAGRPAAFGTVKEDSGSSSPPGVFLEKDYEIYRDYSADGQLLHYRTSSLRWNSGAKERMLIKVADREPSFLAAQGNGYAPDGPPGVRSRRPSGSQHSPSLQTFAPEADGTIFFPERRPSPFLKRAELPGSSSPLLAQPRKPSGDSQPSSPRYGYEPPLYEEPPVEYQAPIYDEPPMDVQFEAGGGYQAGSPQRSPGRKPRPFLQPNKQGPPSPCQQLVLTKQKCPERFLSLEYSPAGKEYVRQLVYVEQAGSSPKLRAGPRHKYAPNPGGGSYSLQPSPCLLRDQRLGVKSGDYSTMEGPELRHSQPPTPLPQAQEDAMSWSSQQDTLSSTGYSPGTRKRKSRKPSLCQATSATPTEGPGDLLVEQPLAEEQPPCGTSLAPVKRAEGEAEGARGAAEPFLAQARLAWEAQQAHFHMKQRSSWDSQQDGSGYESDGALPLPMPGPVVRAFSEDEALAQQENRHWRRGTFEKLGFPQILLEKSVSVQTNLASPEPYLHPSQSEDLAACAQFESSRQSRSGVPSSSCVFPTFTLRKPSSETDIENWASKHFNKHTQGLFRRKVSIANMLAWSSESIKKPMIVTSDRHVKKEACELFKLIQMYMGDRRAKADPLHVALEVATKGWSVQGLRDELYIQLCRQTTENFRLESLARGWELMAICLAFFPPTPKFHSYLEGYIYRHMDPVNDTKGVAISTYAKYCYHKLQKAALTGAKKGLKKPNVEEIRHAKNAVFSPSMFGSALQEVMGMQRERYPERQLPWVQTRLSEEVLALNGDQTEGIFRVPGDIDEVNALKLQVDQWKVPTGLEDPHVPASLLKLWYRELEEPLIPHEFYEQCIAHYDSPEAAVAVVHALPRINRMVLCYLIRFLQVFVQPANVAVTKMDVSNLAMVMAPNCLRCQSDDPRVIFENTRKEMSFLRVLIQHLDTSFMEGVL",
# SRCN1_HUMAN
"MGNAPSQDPERSSPPMLSADDAEYPREYRTLGGGGGGGSGGRRFSNVGLVHTSERRHTVIAAQSLEALSGLQKADADRKRDAFMDHLKSKYPQHALALRGQQDRMREQPNYWSFKTRSSRHTQGAQPGLADQAAKLSYASAESLETMSEAELPLGFSRMNRFRQSLPLSRSASQTKLRSPGVLFLQFGEETRRVHITHEVSSLDTLHALIAHMFPQKLTMGMLKSPNTAILIKDEARNVFYELEDVRDIQDRSIIKIYRKEPLYAAFPGSHLTNGDLRREMVYASRESSPTRRLNNLSPAPHLASGSPPPGLPSGLPSGLQSGSPSRSRLSYAGGRPPSYAGSPVHHAAERLGGAPAAQGVSPSPSAILERRDVKPDEDLASKAGGMVLVKGEGLYADPYGLLHEGRLSLAAAAGDPFAYPGAGGLYKRGSVRSLSTYSAAALQSDLEDSLYKAAGGGGPLYGDGYGFRLPPSSPQKLADVAAPPGGPPPPHSPYSGPPSRGSPVRQSFRKDSGSSSVFAESPGGKTRSAGSASTAGAPPSELFPGPGERSLVGFGPPVPAKDTETRERMEAMEKQIASLTGLVQSALLRGSEPETPSEKIEGSNGAATPSAPCGSGGRSSGATPVSGPPPPSASSTPAGQPTAVSRLQMQLHLRGLQNSASDLRGQLQQLRKLQLQNQESVRALLKRTEAELSMRVSEAARRQEDPLQRQRTLVEEERLRYLNDEELITQQLNDLEKSVEKIQRDVSHNHRLVPGPELEEKALVLKQLGETLTELKAHFPGLQSKMRVVLRVEVEAVKFLKEEPQRLDGLLKRCRGVTDTLAQIRRQVDEGVWPPPNNLLSQSPKKVTAETDFNKSVDFEMPPPSPPLNLHELSGPAEGASLTPKGGNPTKGLDTPGKRSVDKAVSVEAAERDWEEKRAALTQYSAKDINRLLEETQAELLKAIPDLDCASKAHPGPAPTPDHKPPKAPHGQKAAPRTEPSGRRGSDELTVPRYRTEKPSKSPPPPPPRRSFPSSHGLTTTRTGEVVVTSKKDSAFIKKAESEELEVQKPQVKLRRAVSEVARPASTPPIMASAIKDEDDEDRIIAELESGGGSVPPMKVVTPGASRLKAAQGQAGSPDKSKHGKQRAEYMRIQAQQQATKPSKEMSGSNETSSPVSEKPSASRTSIPVLTSFGARNSSISF",
# ROGDI_HUMAN
"MATVMAATAAERAVLEEEFRWLLHDEVHAVLKQLQDILKEASLRFTLPGSGTEGPAKQENFILGSCGTDQVKGVLTLQGDALSQADVNLKMPRNNQLLHFAFREDKQWKLQQIQDARNHVSQAIYLLTSRDQSYQFKTGAEVLKLMDAVMLQLTRARNRLTTPATLTLPEIAASGLTRMFAPALPSDLLVNVYINLNKLCLTVYQLHALQPNSTKNFRPAGGAVLHSPGAMFEWGSQRLEVSHVHKVECVIPWLNDALVYFTVSLQLCQQLKDKISVFSSYWSYRPF",
# MLP3B_HUMAN
"MPSEKTFKQRRTFEQRVEDVRLIREQHPTKIPVIIERYKGEKQLPVLDKTKFLVPDHVNMSELIKIIRRRLQLNANQAFFLLVNGHSMVSVSTPISEVYESEKDEDGFLYMVYASQETFGMKLSV",
# SENP6_HUMAN
"MAAGKSGGSAGEITFLEALARSESKRDGGFKNNWSFDHEEESEGDTDKDGTNLLSVDEDEDSETSKGKKLNRRSEIVANSSGEFILKTYVRRNKSESFKTLKGNPIGLNMLSNNKKLSENTQNTSLCSGTVVHGRRFHHAHAQIPVVKTAAQSSLDRKERKEYPPHVQKVEINPVRLSRLQGVERIMKKTEESESQVEPEIKRKVQQKRHCSTYQPTPPLSPASKKCLTHLEDLQRNCRQAITLNESTGPLLRTSIHQNSGGQKSQNTGLTTKKFYGNNVEKVPIDIIVNCDDSKHTYLQTNGKVILPGAKIPKITNLKERKTSLSDLNDPIILSSDDDDDNDRTNRRESISPQPADSACSSPAPSTGKVEAALNENTCRAERELRSIPEDSELNTVTLPRKARMKDQFGNSIINTPLKRRKVFSQEPPDALALSCQSSFDSVILNCRSIRVGTLFRLLIEPVIFCLDFIKIQLDEPDHDPVEIILNTSDLTKCEWCNVRKLPVVFLQAIPAVYQKLSIQLQMNKEDKVWNDCKGVNKLTNLEEQYIILIFQNGLDPPANMVFESIINEIGIKNNISNFFAKIPFEEANGRLVACTRTYEESIKGSCGQKENKIKTVSFESKIQLRSKQEFQFFDEEEETGENHTIFIGPVEKLIVYPPPPAKGGISVTNEDLHCLNEGEFLNDVIIDFYLKYLVLEKLKKEDADRIHIFSSFFYKRLNQRERRNHETTNLSIQQKRHGRVKTWTRHVDIFEKDFIFVPLNEAAHWFLAVVCFPGLEKPKYEPNPHYHENAVIQKCSTVEDSCISSSASEMESCSQNSSAKPVIKKMLNKKHCIAVIDSNPGQEESDPRYKRNICSVKYSVKKINHTASENEEFNKGESTSQKVADRTKSENGLQNESLSSTHHTDGLSKIRLNYSDESPEAGKMLEDELVDFSEDQDNQDDSSDDGFLADDNCSSEIGQWHLKPTICKQPCILLMDSLRGPSRSNVVKILREYLEVEWEVKKGSKRSFSKDVMKGSNPKVPQQNNFSDCGVYVLQYVESFFENPILSFELPMNLANWFPPPRMRTKREEIRNIILKLQEDQSKEKRKHKDTYSTEAPLGEGTEQYVNSISD",
# EGLN1_HUMAN
"MANDSGGPGGPSPSERDRQYCELCGKMENLLRCSRCRSSFYCCKEHQRQDWKKHKLVCQGSEGALGHGVGPHQHSGPAPPAAVPPPRAGAREPRKAAARRDNASGDAAKGKVKAKPPADPAAAASPCRAAAGGQGSAVAAEAEPGKEEPPARSSLFQEKANLYPPSNTPGDALSPGGGLRPNGQTKPLPALKLALEYIVPCMNKHGICVVDDFLGKETGQQIGDEVRALHDTGKFTDGQLVSQKSDSSKDIRGDKITWIEGKEPGCETIGLLMSSMDDLIRHCNGKLGSYKINGRTKAMVACYPGNGTGYVRHVDNPNGDGRCVTCIYYLNKDWDAKVSGGILRIFPEGKAQFADIEPKFDRLLFFWSDRRNPHEVQPAYATRYAITVWYFDADERARAKVKYLTGEKGVRVELNKPSDSVGKDVF",
# ACH10_HUMAN
"MGLRSHHLSLGLLLLFLLPAECLGAEGRLALKLFRDLFANYTSALRPVADTDQTLNVTLEVTLSQIIDMDERNQVLTLYLWIRQEWTDAYLRWDPNAYGGLDAIRIPSSLVWRPDIVLYNKADAQPPGSASTNVVLRHDGAVRWDAPAITRSSCRVDVAAFPFDAQHCGLTFGSWTHGGHQLDVRPRGAAASLADFVENVEWRVLGMPARRRVLTYGCCSEPYPDVTFTLLLRRRAAAYVCNLLLPCVLISLLAPLAFHLPADSGEKVSLGVTVLLALTVFQLLLAESMPPAESVPLIGKYYMATMTMVTFSTALTILIMNLHYCGPSVRPVPAWARALLLGHLARGLCVRERGEPCGQSRPPELSPSPQSPEGGAGPPAGPCHEPRCLCRQEALLHHVATIANTFRSHRAAQRCHEDWKRLARVMDRFFLAIFFSMALVMSLLVLVQAL",
# RB33B_HUMAN
"MAEEMESSLEASFSSSGAVSGASGFLPPARSRIFKIIVIGDSNVGKTCLTYRFCAGRFPDRTEATIGVDFRERAVEIDGERIKIQLWDTAGQERFRKSMVQHYYRNVHAVVFVYDMTNMASFHSLPSWIEECKQHLLANDIPRILVGNKCDLRSAIQVPTDLAQKFADTHSMPLFETSAKNPNDNDHVEAIFMTLAHKLKSHKPLMLSQPPDNGIILKPEPKPAMTCWC",
# SHRPN_HUMAN
"MAPPAGGAAAAASDLGSAAVLLAVHAAVRPLGAGPDAEAQLRRLQLSADPERPGRFRLELLGAGPGAVNLEWPLESVSYTIRGPTQHELQPPPGGPGTLSLHFLNPQEAQRWAVLVRGATVEGQNGSKSNSPPALGPEACPVSLPSPPEASTLKGPPPEADLPRSPGNLTEREELAGSLARAIAGGDEKGAAQVAAVLAQHRVALSVQLQEACFPPGPIRLQVTLEDAASAASAASSAHVALQVHPHCTVAALQEQVFSELGFPPAVQRWVIGRCLCVPERSLASYGVRQDGDPAFLYLLSAPREAPATGPSPQHPQKMDGELGRLFPPSLGLPPGPQPAASSLPSPLQPSWSCPSCTFINAPDRPGCEMCSTQRPCTWDPLAAAST",
# FXYD6_HUMAN
"MELVLVFLCSLLAPMVLASAAEKEKEMDPFHYDYQTLRIGGLVFAVVLFSVGILLILSRRCKCSFNQKPRAPGDEEAQVENLITANATEPQKAEN",
# RAB17_HUMAN
"MAQAHRTPQPRAAPSQPRVFKLVLLGSGSVGKSSLALRYVKNDFKSILPTVGCAFFTKVVDVGATSLKLEIWDTAGQEKYHSVCHLYFRGANAALLVYDITRKDSFLKAQQWLKDLEEELHPGEVLVMLVGNKTDLSQEREVTFQEGKEFADSQKLLFMETSAKLNHQVSEVFNTVAQELLQRSDEEGQALRGDAAVALNKGPARQAKCCAH",
# RAB1B_HUMAN
"MNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVDFKIRTIELDGKTIKLQIWDTAGQERFRTITSSYYRGAHGIIVVYDVTDQESYANVKQWLQEIDRYASENVNKLLVGNKSDLTTKKVVDNTTAKEFADSLGIPFLETSAKNATNVEQAFMTMAAEIKKRMGPGAASGGERPNLKIDSTPVKPAGGGCC",
# SNAB_HUMAN
"MDNAGKEREAVQLMAEAEKRVKASHSFLRGLFGGNTRIEEACEMYTRAANMFKMAKNWSAAGNAFCQAAKLHMQLQSKHDSATSFVDAGNAYKKADPQEAINCLNAAIDIYTDMGRFTIAAKHHITIAEIYETELVDIEKAIAHYEQSADYYKGEESNSSANKCLLKVAAYAAQLEQYQKAIEIYEQVGANTMDNPLLKYSAKDYFFKAALCHFIVDELNAKLALEKYEEMFPAFTDSRECKLLKKLLEAHEEQNSEAYTEAVKEFDSISRLDQWLTTMLLRIKKSIQGDGEGDGDLK",
# SLIK2_HUMAN
"MLSGVWFLSVLTVAGILQTESRKTAKDICKIRCLCEEKENVLNINCENKGFTTVSLLQPPQYRIYQLFLNGNLLTRLYPNEFVNYSNAVTLHLGNNGLQEIRTGAFSGLKTLKRLHLNNNKLEILREDTFLGLESLEYLQADYNYISAIEAGAFSKLNKLKVLILNDNLLLSLPSNVFRFVLLTHLDLRGNRLKVMPFAGVLEHIGGIMEIQLEENPWNCTCDLLPLKAWLDTITVFVGEIVCETPFRLHGKDVTQLTRQDLCPRKSASDSSQRGSHADTHVQRLSPTMNPALNPTRAPKASRPPKMRNRPTPRVTVSKDRQSFGPIMVYQTKSPVPLTCPSSCVCTSQSSDNGLNVNCQERKFTNISDLQPKPTSPKKLYLTGNYLQTVYKNDLLEYSSLDLLHLGNNRIAVIQEGAFTNLTSLRRLYLNGNYLEVLYPSMFDGLQSLQYLYLEYNVIKEIKPLTFDALINLQLLFLNNNLLRSLPDNIFGGTALTRLNLRNNHFSHLPVKGVLDQLPAFIQIDLQENPWDCTCDIMGLKDWTEHANSPVIINEVTCESPAKHAGEILKFLGREAICPDSPNLSDGTVLSMNHNTDTPRSLSVSPSSYPELHTEVPLSVLILGLLVVFILSVCFGAGLFVFVLKRRKGVPSVPRNTNNLDVSSFQLQYGSYNTETHDKTDGHVYNYIPPPVGQMCQNPIYMQKEGDPVAYYRNLQEFSYSNLEEKKEEPATPAYTISATELLEKQATPREPELLYQNIAERVKELPSAGLVHYNFCTLPKRQFAPSYESRRQNQDRINKTVLYGTPRKCFVGQSKPNHPLLQAKPQSEPDYLEVLEKQTAISQL",
# BC11A_HUMAN
"MSRRKQGKPQHLSKREFSPEPLEAILTDDEPDHGPLGAPEGDHDLLTCGQCQMNFPLGDILIFIEHKRKQCNGSLCLEKAVDKPPSPSPIEMKKASNPVEVGIQVTPEDDDCLSTSSRGICPKQEHIADKLLHWRGLSSPRSAHGALIPTPGMSAEYAPQGICKDEPSSYTCTTCKQPFTSAWFLLQHAQNTHGLRIYLESEHGSPLTPRVGIPSGLGAECPSQPPLHGIHIADNNPFNLLRIPGSVSREASGLAEGRFPPTPPLFSPPPRHHLDPHRIERLGAEEMALATHHPSAFDRVLRLNPMAMEPPAMDFSRRLRELAGNTSSPPLSPGRPSPMQRLLQPFQPGSKPPFLATPPLPPLQSAPPPSQPPVKSKSCEFCGKTFKFQSNLVVHRRSHTGEKPYKCNLCDHACTQASKLKRHMKTHMHKSSPMTVKSDDGLSTASSPEPGTSDLVGSASSALKSVVAKFKSENDPNLIPENGDEEEEEDDEEEEEEEEEEEEELTESERVDYGFGLSLEAARHHENSSRGAVVGVGDESRALPDVMQGMVLSSMQHFSEAFHQVLGEKHKRGHLAEAEGHRDTCDEDSVAGESDRIDDGTVNGRGCSPGESASGGLSKKLLLGSPSSLSPFSKRIKLEKEFDLPPAAMPNTENVYSQWLAGYAASRQLKDPFLSFGDSRQSPFASSSEHSSENGSLRFSTPPGELDGGISGRSGTGSGGSTPHISGPGPGRPSSKEGRRSDTCEYCGKVFKNCSNLTVHRRSHTGERPYKCELCNYACAQSSKLTRHMKTHGQVGKDVYKCEICKMPFSVYSTLEKHMKKWHSDRVLNNDIKTE",
# OSBL2_HUMAN
"MNGEEEFFDAVTGFDSDNSSGEFSEANQKVTGMIDLDTSKNNRIGKTGERPSQENGIQKHRTSLPAPMFSRSDFSVWTILKKCVGLELSKITMPIAFNEPLSFLQRITEYMEHVYLIHRASCQPQPLERMQSVAAFAVSAVASQWERTGKPFNPLLGETYELIREDLGFRFISEQVSHHPPISAFHSEGLNHDFLFHGSIYPKLKFWGKSVEAEPRGTITLELLKHNEAYTWTNPTCCVHNVIIGKLWIEQYGTVEILNHRTGHKCVLHFKPCGLFGKELHKVEGHIQDKNKKKLFMIYGKWTECLWGIDPVSYESFKKQERRGDHLRKAKLDEDSGKADSDVADDVPVAQETVQVIPGSKLLWRINTRPPNSAQMYNFTSFTVSLNELETGMEKTLPPTDCRLRPDIRGMENGNMDLASQEKERLEEKQREARRERAKEEAEWQTRWFYPGNNPYTGTPDWLYAGDYFERNFSDCPDIY",
# S6A17_HUMAN
"MPKNSKVTQREHSSEHVTESVADLLALEEPVDYKQSVLNVAGEAGGKQKAVEEELDAEDRPAWNSKLQYILAQIGFSVGLGNIWRFPYLCQKNGGGAYLVPYLVLLIIIGIPLFFLELAVGQRIRRGSIGVWHYICPRLGGIGFSSCIVCLFVGLYYNVIIGWSIFYFFKSFQYPLPWSECPVVRNGSVAVVEAECEKSSATTYFWYREALDISDSISESGGLNWKMTLCLLVAWSIVGMAVVKGIQSSGKVMYFSSLFPYVVLACFLVRGLLLRGAVDGILHMFTPKLDKMLDPQVWREAATQVFFALGLGFGGVIAFSSYNKQDNNCHFDAALVSFINFFTSVLATLVVFAVLGFKANIMNEKCVVENAEKILGYLNTNVLSRDLIPPHVNFSHLTTKDYMEMYNVIMTVKEDQFSALGLDPCLLEDELDKSVQGTGLAFIAFTEAMTHFPASPFWSVMFFLMLINLGLGSMIGTMAGITTPIIDTFKVPKEMFTVGCCVFAFLVGLLFVQRSGNYFVTMFDDYSATLPLTLIVILENIAVAWIYGTKKFMQELTEMLGFRPYRFYFYMWKFVSPLCMAVLTTASIIQLGVTPPGYSAWIKEEAAERYLYFPNWAMALLITLIVVATLPIPVVFVLRHFHLLSDGSNTLSVSYKKGRMMKDISNLEENDETRFILSKVPSEAPSPMPTHRSYLGPGSTSPLETSGNPNGRYGSGYLLASTPESEL",
# ATG5_HUMAN
"MTDDKDVLRDVWFGRIPTCFTLYQDEITEREAEPYYLLLPRVSYLTLVTDKVKKHFQKVMRQEDISEIWFEYEGTPLKWHYPIGLLFDLLASSSALPWNITVHFKSFPEKDLLHCPSKDAIEAHFMSCMKEADALKHKSQVINEMQKKDHKQLWMGLQNDRFDQFWAINRKLMEYPAEENGFRYIPFRIYQTTTERPFIQKLFRPVAADGQLHTLGDLLKEVCPSAIDPEDGEKKNQVMIHGIEPMLETPLQWLSEHLSYPDNFLHISIIPQPTD",
# S1PR5_HUMAN
"MESGLLRPAPVSEVIVLHYNYTGKLRGARYQPGAGLRADAVVCLAVCAFIVLENLAVLLVLGRHPRFHAPMFLLLGSLTLSDLLAGAAYAANILLSGPLTLKLSPALWFAREGGVFVALTASVLSLLAIALERSLTMARRGPAPVSSRGRTLAMAAAAWGVSLLLGLLPALGWNCLGRLDACSTVLPLYAKAYVLFCVLAFVGILAAICALYARIYCQVRANARRLPARPGTAGTTSTRARRKPRSLALLRTLSVVLLAFVACWGPLFLLLLLDVACPARTCPVLLQADPFLGLAMANSLLNPIIYTLTNRDLRHALLRLVCCGRHSCGRDPSGSQQSASAAEASGGLRRCLPPGLDGSFSGSERSSPQRDGLDTSGSTGSPGAPTAARTLVSEPAAD",
# PORCN_HUMAN
"MATFSRQEFFQQLLQGCLLPTAQQGLDQIWLLLAICLACRLLWRLGLPSYLKHASTVAGGFFSLYHFFQLHMVWVVLLSLLCYLVLFLCRHSSHRGVFLSVTILIYLLMGEMHMVDTVTWHKMRGAQMIVAMKAVSLGFDLDRGEVGTVPSPVEFMGYLYFVGTIVFGPWISFHSYLQAVQGRPLSCRWLQKVARSLALALLCLVLSTCVGPYLFPYFIPLNGDRLLRNKKRKARGTMVRWLRAYESAVSFHFSNYFVGFLSEATATLAGAGFTEEKDHLEWDLTVSKPLNVELPRSMVEVVTSWNLPMSYWLNNYVFKNALRLGTFSAVLVTYAASALLHGFSFHLAAVLLSLAFITYVEHVLRKRLARILSACVLSKRCPPDCSHQHRLGLGVRALNLLFGALAIFHLAYLGSLFDVDVDDTTEEQGYGMAYTVHKWSELSWASHWVTFGCWIFYRLIG",
# CAD23_HUMAN
"MGRHVATSCHVAWLLVLISGCWGQVNRLPFFTNHFFDTYLLISEDTPVGSSVTQLLAQDMDNDPLVFGVSGEEASRFFAVEPDTGVVWLRQPLDRETKSEFTVEFSVSDHQGVITRKVNIQVGDVNDNAPTFHNQPYSVRIPENTPVGTPIFIVNATDPDLGAGGSVLYSFQPPSQFFAIDSARGIVTVIRELDYETTQAYQLTVNATDQDKTRPLSTLANLAIIITDVQDMDPIFINLPYSTNIYEHSPPGTTVRIITAIDQDKGRPRGIGYTIVSGNTNSIFALDYISGVLTLNGLLDRENPLYSHGFILTVKGTELNDDRTPSDATVTTTFNILVIDINDNAPEFNSSEYSVAITELAQVGFALPLFIQVVDKDENLGLNSMFEVYLVGNNSHHFIISPTSVQGKADIRIRVAIPLDYETVDRYDFDLFANESVPDHVGYAKVKITLINENDNRPIFSQPLYNISLYENVTVGTSVLTVLATDNDAGTFGEVSYFFSDDPDRFSLDKDTGLIMLIARLDYELIQRFTLTIIARDGGGEETTGRVRINVLDVNDNVPTFQKDAYVGALRENEPSVTQLVRLRATDEDSPPNNQITYSIVSASAFGSYFDISLYEGYGVISVSRPLDYEQISNGLIYLTVMAMDAGNPPLNSTVPVTIEVFDENDNPPTFSKPAYFVSVVENIMAGATVLFLNATDLDRSREYGQESIIYSLEGSTQFRINARSGEITTTSLLDRETKSEYILIVRAVDGGVGHNQKTGIATVNITLLDINDNHPTWKDAPYYINLVEMTPPDSDVTTVVAVDPDLGENGTLVYSIQPPNKFYSLNSTTGKIRTTHAMLDRENPDPHEAELMRKIVVSVTDCGRPPLKATSSATVFVNLLDLNDNDPTFQNLPFVAEVLEGIPAGVSIYQVVAIDLDEGLNGLVSYRMPVGMPRMDFLINSSSGVVVTTTELDRERIAEYQLRVVASDAGTPTKSSTSTLTIHVLDVNDETPTFFPAVYNVSVSEDVPREFRVVWLNCTDNDVGLNAELSYFITGGNVDGKFSVGYRDAVVRTVVGLDRETTAAYMLILEAIDNGPVGKRHTGTATVFVTVLDVNDNRPIFLQSSYEASVPEDIPEGHSILQLKATDADEGEFGRVWYRILHGNHGNNFRIHVSNGLLMRGPRPLDRERNSSHVLIVEAYNHDLGPMRSSVRVIVYVEDINDEAPVFTQQQYSRLGLRETAGIGTSVIVVQATDRDSGDGGLVNYRILSGAEGKFEIDESTGLIITVNYLDYETKTSYMMNVSATDQAPPFNQGFCSVYITLLNELDEAVQFSNASYEAAILENLALGTEIVRVQAYSIDNLNQITYRFNAYTSTQAKALFKIDAITGVITVQGLVDREKGDFYTLTVVADDGGPKVDSTVKVYITVLDENDNSPRFDFTSDSAVSIPEDCPVGQRVATVKAWDPDAGSNGQVVFSLASGNIAGAFEIVTTNDSIGEVFVARPLDREELDHYILQVVASDRGTPPRKKDHILQVTILDINDNPPVIESPFGYNVSVNENVGGGTAVVQVRATDRDIGINSVLSYYITEGNKDMAFRMDRISGEIATRPAPPDRERQSFYHLVATVEDEGTPTLSATTHVYVTIVDENDNAPMFQQPHYEVLLDEGPDTLNTSLITIQALDLDEGPNGTVTYAIVAGNIVNTFRIDRHMGVITAAKELDYEISHGRYTLIVTATDQCPILSHRLTSTTTVLVNVNDINDNVPTFPRDYEGPFEVTEGQPGPRVWTFLAHDRDSGPNGQVEYSIMDGDPLGEFVISPVEGVLRVRKDVELDRETIAFYNLTICARDRGMPPLSSTMLVGIRVLDINDNDPVLLNLPMNITISENSPVSSFVAHVLASDADSGCNARLTFNITAGNRERAFFINATTGIVTVNRPLDRERIPEYKLTISVKDNPENPRIARRDYDLLLIFLSDENDNHPLFTKSTYQAEVMENSPAGTPLTVLNGPILALDADQDIYAVVTYQLLGAQSGLFDINSSTGVVTVRSGVIIDREAFSPPILELLLLAEDIGLLNSTAHLLITILDDNDNRPTFSPATLTVHLLENCPPGFSVLQVTATDEDSGLNGELVYRIEAGAQDRFLIHLVTGVIRVGNATIDREEQESYRLTVVATDRGTVPLSGTAIVTILIDDINDSRPEFLNPIQTVSVLESAEPGTVIANITAIDHDLNPKLEYHIVGIVAKDDTDRLVPNQEDAFAVNINTGSVMVKSPMNRELVATYEVTLSVIDNASDLPERSVSVPNAKLTVNVLDVNDNTPQFKPFGITYYMERILEGATPGTTLIAVAAVDPDKGLNGLVTYTLLDLVPPGYVQLEDSSAGKVIANRTVDYEEVHWLNFTVRASDNGSPPRAAEIPVYLEIVDINDNNPIFDQPSYQEAVFEDVPVGTIILTVTATDADSGNFALIEYSLGDGESKFAINPTTGDIYVLSSLDREKKDHYILTALAKDNPGDVASNRRENSVQVVIQVLDVNDCRPQFSKPQFSTSVYENEPAGTSVITMMATDQDEGPNGELTYSLEGPGVEAFHVDMDSGLVTTQRPLQSYEKFSLTVVATDGGEPPLWGTTMLLVEVIDVNDNRPVFVRPPNGTILHIREEIPLRSNVYEVYATDKDEGLNGAVRYSFLKTAGNRDWEFFIIDPISGLIQTAQRLDRESQAVYSLILVASDLGQPVPYETMQPLQVALEDIDDNEPLFVRPPKGSPQYQLLTVPEHSPRGTLVGNVTGAVDADEGPNAIVYYFIAAGNEEKNFHLQPDGCLLVLRDLDREREAIFSFIVKASSNRSWTPPRGPSPTLDLVADLTLQEVRVVLEDINDQPPRFTKAEYTAGVATDAKVGSELIQVLALDADIGNNSLVFYSILAIHYFRALANDSEDVGQVFTMGSMDGILRTFDLFMAYSPGYFVVDIVARDLAGHNDTAIIGIYILRDDQRVKIVINEIPDRVRGFEEEFIHLLSNITGAIVNTDNVQFHVDKKGRVNFAQTELLIHVVNRDTNRILDVDRVIQMIDENKEQLRNLFRNYNVLDVQPAISVRLPDDMSALQMAIIVLAILLFLAAMLFVLMNWYYRTVHKRKLKAIVAGSAGNRGFIDIMDMPNTNKYSFDGANPVWLDPFCRNLELAAQAEHEDDLPENLSEIADLWNSPTRTHGTFGREPAAVKPDDDRYLRAAIQEYDNIAKLGQIIREGPIKGSLLKVVLEDYLRLKKLFAQRMVQKASSCHSSISELIQTELDEEPGDHSPGQGSLRFRHKPPVELKGPDGIHVVHGSTGTLLATDLNSLPEEDQKGLGRSLETLTAAEATAFERNARTESAKSTPLHKLRDVIMETPLEITEL",
# VP33B_HUMAN
"MAFPHRPDAPELPDFSMLKRLARDQLIYLLEQLPGKKDLFIEADLMSPLDRIANVSILKQHEVDKLYKVENKPALSSNEQLCFLVRPRIKNMRYIASLVNADKLAGRTRKYKVIFSPQKFYACEMVLEEEGIYGDVSCDEWAFSLLPLDVDLLSMELPEFFRDYFLEGDQRWINTVAQALHLLSTLYGPFPNCYGIGRCAKMAYELWRNLEEEEDGETKGRRPEIGHIFLLDRDVDFVTALCSQVVYEGLVDDTFRIKCGSVDFGPEVTSSDKSLKVLLNAEDKVFNEIRNEHFSNVFGFLSQKARNLQAQYDRRRGMDIKQMKNFVSQELKGLKQEHRLLSLHIGACESIMKKKTKQDFQELIKTEHALLEGFNIRESTSYIEEHIDRQVSPIESLRLMCLLSITENGLIPKDYRSLKTQYLQSYGPEHLLTFSNLRRAGLLTEQAPGDTLTAVESKVSKLVTDKAAGKITDAFSSLAKRSNFRAISKKLNLIPRVDGEYDLKVPRDMAYVFGGAYVPLSCRIIEQVLERRSWQGLDEVVRLLNCSDFAFTDMTKEDKASSESLRLILVVFLGGCTFSEISALRFLGREKGYRFIFLTTAVTNSARLMEAMSEVKA",
# VPS16_HUMAN
"MDCYTANWNPLGDSAFYRKYELYSMDWDLKEELRDCLVAAAPYGGPIALLRNPWRKEKAASVRPVLDIYSASGMPLASLLWKSGPVVSLGWSAEEELLCVQEDGAVLVYGLHGDFRRHFSMGNEVLQNRVLDARIFHTEFGSGVAILTGAHRFTLSANVGDLKLRRMPEVPGLQSAPSCWTVLCQDRVAHILLAVGPDLYLLDHAACSAVTPPGLAPGVSSFLQMAVSFTYRHLALFTDTGYIWMGTASLKEKLCEFNCNIRAPPKQMVWCSRPRSKERAVVVAWERRLMVVGDAPESIQFVLDEDSYLVPELDGVRIFSRSTHEFLHEVPAASEEIFKIASMAPGALLLEAQKEYEKESQKADEYLREIQELGQLTQAVQQCIEAAGHEHQPDMQKSLLRAASFGKCFLDRFPPDSFVHMCQDLRVLNAVRDYHIGIPLTYSQYKQLTIQVLLDRLVLRRLYPLAIQICEYLRLPEVQGVSRILAHWACYKVQQKDVSDEDVARAINQKLGDTPGVSYSDIAARAYGCGRTELAIKLLEYEPRSGEQVPLLLKMKRSKLALSKAIESGDTDLVFTVLLHLKNELNRGDFFMTLRNQPMALSLYRQFCKHQELETLKDLYNQDDNHQELGSFHIRASYAAEERIEGRVAALQTAADAFYKAKNEFAAKATEDQMRLLRLQRRLEDELGGQFLDLSLHDTVTTLILGGHNKRAEQLARDFRIPDKRLWWLKLTALADLEDWEELEKFSKSKKSPIGYLPFVEICMKQHNKYEAKKYASRVGPEQKVKALLLVGDVAQAADVAIEHRNEAELSLVLSHCTGATDGATADKIQRARAQAQKK",
# VPS11_HUMAN
"MAAYLQWRRFVFFDKELVKEPLSNDGAAPGATPASGSAASKFLCLPPGITVCDSGRGSLVFGDMEGQIWFLPRSLQLTGFQAYKLRVTHLYQLKQHNILASVGEDEEGINPLVKIWNLEKRDGGNPLCTRIFPAIPGTEPTVVSCLTVHENLNFMAIGFTDGSVTLNKGDITRDRHSKTQILHKGNYPVTGLAFRQAGKTTHLFVVTTENVQSYIVSGKDYPRVELDTHGCGLRCSALSDPSQDLQFIVAGDECVYLYQPDERGPCFAFEGHKLIAHWFRGYLIIVSRDRKVSPKSEFTSRDSQSSDKQILNIYDLCNKFIAYSTVFEDVVDVLAEWGSLYVLTRDGRVHALQEKDTQTKLEMLFKKNLFEMAINLAKSQHLDSDGLAQIFMQYGDHLYSKGNHDGAVQQYIRTIGKLEPSYVIRKFLDAQRIHNLTAYLQTLHRQSLANADHTTLLLNCYTKLKDSSKLEEFIKKKSESEVHFDVETAIKVLRQAGYYSHALYLAENHAHHEWYLKIQLEDIKNYQEALRYIGKLPFEQAESNMKRYGKILMHHIPEQTTQLLKGLCTDYRPSLEGRSDREAPGCRANSEEFIPIFANNPRELKAFLEHMSEVQPDSPQGIYDTLLELRLQNWAHEKDPQVKEKLHAEAISLLKSGRFCDVFDKALVLCQMHDFQDGVLYLYEQGKLFQQIMHYHMQHEQYRQVISVCERHGEQDPSLWEQALSYFARKEEDCKEYVAAVLKHIENKNLMPPLLVVQTLAHNSTATLSVIRDYLVQKLQKQSQQIAQDELRVRRYREETTRIRQEIQELKASPKIFQKTKCSICNSALELPSVHFLCGHSFHQHCFESYSESDADCPTCLPENRKVMDMIRAQEQKRDLHDQFQHQLRCSNDSFSVIADYFGRGVFNKLTLLTDPPTARLTSSLEAGLQRDLLMHSRRGT",
# SYT4_HUMAN
"MAPITTSREEFDEIPTVVGIFSAFGLVFTVSLFAWICCQRKSSKSNKTPPYKFVHVLKGVDIYPENLNSKKKFGADDKNEVKNKPAVPKNSLHLDLEKRDLNGNFPKTNLKPGSPSDLENATPKLFLEGEKESVSPESLKSSTSLTSEEKQEKLGTLFFSLEYNFERKAFVVNIKEARGLPAMDEQSMTSDPYIKMTILPEKKHKVKTRVLRKTLDPAFDETFTFYGIPYTQIQELALHFTILSFDRFSRDDIIGEVLIPLSGIELSEGKMLMNREIIKRNVRKSSGRGELLISLCYQSTTNTLTVVVLKARHLPKSDVSGLSDPYVKVNLYHAKKRISKKKTHVKKCTPNAVFNELFVFDIPCEGLEDISVEFLVLDSERGSRNEVIGQLVLGAAAEGTGGEHWKEICDYPRRQIAKWHVLCDG",
# KCNN2_HUMAN
"MSSCRYNGGVMRPLSNLSASRRNLHEMDSEAQPLQPPASVGGGGGASSPSAAAAAAAAVSSSAPEIVVSKPEHNNSNNLALYGTGGGGSTGGGGGGGGSGHGSSSGTKSSKKKNQNIGYKLGHRRALFEKRKRLSDYALIFGMFGIVVMVIETELSWGAYDKASLYSLALKCLISLSTIILLGLIIVYHAREIQLFMVDNGADDWRIAMTYERIFFICLEILVCAIHPIPGNYTFTWTARLAFSYAPSTTTADVDIILSIPMFLRLYLIARVMLLHSKLFTDASSRSIGALNKINFNTRFVMKTLMTICPGTVLLVFSISLWIIAAWTVRACERYHDQQDVTSNFLGAMWLISITFLSIGYGDMVPNTYCGKGVCLLTGIMGAGCTALVVAVVARKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTKLVKKIDHAKVRKHQRKFLQAIHQLRSVKMEQRKLNDQANTLVDLAKTQNIMYDMISDLNERSEDFEKRIVTLETKLETLIGSIHALPGLISQTIRQQQRDFIEAQMESYDKHVTYNAERSRSSSRRRRSSSTAPPTSSESS",
# IPYR2_HUMAN
"MSALLRLLRTGAPAAACLRLGTSAGTGSRRAMALYHTEERGQPCSQNYRLFFKNVTGHYISPFHDIPLKVNSKEENGIPMKKARNDEYENLFNMIVEIPRWTNAKMEIATKEPMNPIKQYVKDGKLRYVANIFPYKGYIWNYGTLPQTWEDPHEKDKSTNCFGDNDPIDVCEIGSKILSCGEVIHVKILGILALIDEGETDWKLIAINANDPEASKFHDIDDVKKFKPGYLEATLNWFRLYKVPDGKPENQFAFNGEFKNKAFALEVIKSTHQCWKALLMKKCNGGAINCTNVQISDSPFRCTQEEARSLVESVSSSPNKESNEEEQVWHFLGK",
# S12A5_HUMAN
"MSRRFTVTSLPPAGPARSPDPESRRHSVADPRHLPGEDVKGDGNPKESSPFINSTDTEKGKEYDGKNMALFEEEMDTSPMVSSLLSGLANYTNLPQGSREHEEAENNEGGKKKPVQAPRMGTFMGVYLPCLQNIFGVILFLRLTWVVGIAGIMESFCMVFICCSCTMLTAISMSAIATNGVVPAGGSYYMISRSLGPEFGGAVGLCFYLGTTFAGAMYILGTIEILLAYLFPAMAIFKAEDASGEAAAMLNNMRVYGTCVLTCMATVVFVGVKYVNKFALVFLGCVILSILAIYAGVIKSAFDPPNFPICLLGNRTLSRHGFDVCAKLAWEGNETVTTRLWGLFCSSRFLNATCDEYFTRNNVTEIQGIPGAASGLIKENLWSSYLTKGVIVERSGMTSVGLADGTPIDMDHPYVFSDMTSYFTLLVGIYFPSVTGIMAGSNRSGDLRDAQKSIPTGTILAIATTSAVYISSVVLFGACIEGVVLRDKFGEAVNGNLVVGTLAWPSPWVIVIGSFFSTCGAGLQSLTGAPRLLQAISRDGIVPFLQVFGHGKANGEPTWALLLTACICEIGILIASLDEVAPILSMFFLMCYMFVNLACAVQTLLRTPNWRPRFRYYHWTLSFLGMSLCLALMFICSWYYALVAMLIAGLIYKYIEYRGAEKEWGDGIRGLSLSAARYALLRLEEGPPHTKNWRPQLLVLVRVDQDQNVVHPQLLSLTSQLKAGKGLTIVGSVLEGTFLENHPQAQRAEESIRRLMEAEKVKGFCQVVISSNLRDGVSHLIQSGGLGGLQHNTVLVGWPRNWRQKEDHQTWRNFIELVRETTAGHLALLVTKNVSMFPGNPERFSEGSIDVWWIVHDGGMLMLLPFLLRHHKVWRKCKMRIFTVAQMDDNSIQMKKDLTTFLYHLRITAEVEVVEMHESDISAYTYEKTLVMEQRSQILKQMHLTKNEREREIQSITDESRGSIRRKNPANTRLRLNVPEETAGDSEEKPEEEVQLIHDQSAPSCPSSSPSPGEEPEGEGETDPEKVHLTWTKDKSVAEKNKGPSPVSSEGIKDFFSMKPEWENLNQSNVRRMHTAVRLNEVIVKKSRDAKLVLLNMPGPPRNRNGDENYMEFLEVLTEHLDRVMLVRGGGREVITIYS",
# TTYH1_HUMAN
"MGAPPGYRPSAWVHLLHQLPRADFQLRPVPSVFAPQEQEYQQALLLVAALAGLGLGLSLIFIAVYLIRFCCCRPPEPPGSKIPSPGGGCVTWSCIVALLAGCTGIGIGFYGNSETSDGVSQLSSALLHANHTLSTIDHLVLETVERLGEAVRTELTTLEEVLEPRTELVAAARGARRQAEAAAQQLQGLAFWQGVPLSPLQVAENVSFVEEYRWLAYVLLLLLELLVCLFTLLGLAKQSKWLVIVMTVMSLLVLVLSWGSMGLEAATAVGLSDFCSNPDPYVLNLTQEETGLSSDILSYYLLCNRAVSNPFQQRLTLSQRALANIHSQLLGLEREAVPQFPSAQKPLLSLEETLNVTEGNFHQLVALLHCRSLHKDYGAALRGLCEDALEGLLFLLLFSLLSAGALATALCSLPRAWALFPPSDDYDDTDDDDPFNPQESKRFVQWQSSI",
# PTN23_HUMAN
"MEAVPRMPMIWLDLKEAGDFHFQPAVKKFVLKNYGENPEAYNEELKKLELLRQNAVRVPRDFEGCSVLRKYLGQLHYLQSRVPMGSGQEAAVPVTWTEIFSGKSVAHEDIKYEQACILYNLGALHSMLGAMDKRVSEEGMKVSCTHFQCAAGAFAYLREHFPQAYSVDMSRQILTLNVNLMLGQAQECLLEKSMLDNRKSFLVARISAQVVDYYKEACRALENPDTASLLGRIQKDWKKLVQMKIYYFAAVAHLHMGKQAEEQQKFGERVAYFQSALDKLNEAIKLAKGQPDTVQDALRFTMDVIGGKYNSAKKDNDFIYHEAVPALDTLQPVKGAPLVKPLPVNPTDPAVTGPDIFAKLVPMAAHEASSLYSEEKAKLLREMMAKIEDKNEVLDQFMDSMQLDPETVDNLDAYSHIPPQLMEKCAALSVRPDTVRNLVQSMQVLSGVFTDVEASLKDIRDLLEEDELLEQKFQEAVGQAGAISITSKAELAEVRREWAKYMEVHEKASFTNSELHRAMNLHVGNLRLLSGPLDQVRAALPTPALSPEDKAVLQNLKRILAKVQEMRDQRVSLEQQLRELIQKDDITASLVTTDHSEMKKLFEEQLKKYDQLKVYLEQNLAAQDRVLCALTEANVQYAAVRRVLSDLDQKWNSTLQTLVASYEAYEDLMKKSQEGRDFYADLESKVAALLERTQSTCQAREAARQQLLDRELKKKPPPRPTAPKPLLPRREESEAVEAGDPPEELRSLPPDMVAGPRLPDTFLGSATPLHFPPSPFPSSTGPGPHYLSGPLPPGTYSGPTQLIQPRAPGPHAMPVAPGPALYPAPAYTPELGLVPRSSPQHGVVSSPYVGVGPAPPVAGLPSAPPPQFSGPELAMAVRPATTTVDSIQAPIPSHTAPRPNPTPAPPPPCFPVPPPQPLPTPYTYPAGAKQPIPAQHHFSSGIPAGFPAPRIGPQPQPHPQPHPSQAFGPQPPQQPLPLQHPHLFPPQAPGLLPPQSPYPYAPQPGVLGQPPPPLHTQLYPGPAQDPLPAHSGALPFPSPGPPQPPHPPLAYGPAPSTRPMGPQAAPLTIRGPSSAGQSTPSPHLVPSPAPSPGPGPVPPRPPAAEPPPCLRRGAAAADLLSSSPESQHGGTQSPGGGQPLLQPTKVDAAEGRRPQALRLIERDPYEHPERLRQLQQELEAFRGQLGDVGALDTVWRELQDAQEHDARGRSIAIARCYSLKNRHQDVMPYDSNRVVLRSGKDDYINASCVEGLSPYCPPLVATQAPLPGTAADFWLMVHEQKVSVIVMLVSEAEMEKQKVARYFPTERGQPMVHGALSLALSSVRSTETHVERVLSLQFRDQSLKRSLVHLHFPTWPELGLPDSPSNLLRFIQEVHAHYLHQRPLHTPIIVHCSSGVGRTGAFALLYAAVQEVEAGNGIPELPQLVRRMRQQRKHMLQEKLHLRFCYEAVVRHVEQVLQRHGVPPPCKPLASASISQKNHLPQDSQDLVLGGDVPISSIQATIAKLSIRPPGGLESPVASLPGPAEPPGLPPASLPESTPIPSSSPPPLSSPLPEAPQPKEEPPVPEAPSSGPPSSSLELLASLTPEAFSLDSSLRGKQRMSKHNFLQAHNGQGLRATRPSDDPLSLLDPLWTLNKT",
# DNJC5_HUMAN
"MADQRQRSLSTSGESLYHVLGLDKNATSDDIKKSYRKLALKYHPDKNPDNPEAADKFKEINNAHAILTDATKRNIYDKYGSLGLYVAEQFGEENVNTYFVLSSWWAKALFVFCGLLTCCYCCCCLCCCFNCCCGKCKPKAPEGEETEFYVSPEDLEAQLQSDEREATDTPIVIQPASATETTQLTADSHPSYHTDGFN",
# RIMS4_HUMAN
"MERSQSRLSLSASFEALAIYFPCMNSFDDEDAGDSRRLKGAIQRSTETGLAVEMPSRTLRQASHESIEDSMNSYGSEGNLNYGGVCLASDAQFSDFLGSMGPAQFVGRQTLATTPMGDVEIGLQERNGQLEVDIIQARGLTAKPGSKTLPAAYIKAYLLENGICIAKKKTKVARKSLDPLYNQVLLFPESPQGKVLQVIVWGNYGRMERKQFMGVARVLLEELDLTTLAVGWYKLFPTSSMVDPATGPLLRQASQLSLESTVGPCGERS",
# CHM4B_HUMAN
"MSVFGKLFGAGGGKAGKGGPTPQEAIQRLRDTEEMLSKKQEFLEKKIEQELTAAKKHGTKNKRAALQALKRKKRYEKQLAQIDGTLSTIEFQREALENANTNTEVLKNMGYAAKAMKAAHDNMDIDKVDELMQDIADQQELAEEISTAISKPVGFGEEFDEDELMAELEELEQEELDKNLLEISGPETVPLPNVPSIALPSKPAKKKEEEDDDMKELENWAGSM",
# FZD8_HUMAN
"MEWGYLLEVTSLLAALALLQRSSGAAAASAKELACQEITVPLCKGIGYNYTYMPNQFNHDTQDEAGLEVHQFWPLVEIQCSPDLKFFLCSMYTPICLEDYKKPLPPCRSVCERAKAGCAPLMRQYGFAWPDRMRCDRLPEQGNPDTLCMDYNRTDLTTAAPSPPRRLPPPPPGEQPPSGSGHGRPPGARPPHRGGGRGGGGGDAAAPPARGGGGGGKARPPGGGAAPCEPGCQCRAPMVSVSSERHPLYNRVKTGQIANCALPCHNPFFSQDERAFTVFWIGLWSVLCFVSTFATVSTFLIDMERFKYPERPIIFLSACYLFVSVGYLVRLVAGHEKVACSGGAPGAGGAGGAGGAAAGAGAAGAGAGGPGGRGEYEELGAVEQHVRYETTGPALCTVVFLLVYFFGMASSIWWVILSLTWFLAAGMKWGNEAIAGYSQYFHLAAWLVPSVKSIAVLALSSVDGDPVAGICYVGNQSLDNLRGFVLAPLVIYLFIGTMFLLAGFVSLFRIRSVIKQQDGPTKTHKLEKLMIRLGLFTVLYTVPAAVVVACLFYEQHNRPRWEATHNCPCLRDLQPDQARRPDYAVFMLKYFMCLVVGITSGVWVWSGKTLESWRSLCTRCCWASKGAAVGGGAGATAAGGGGGPGGGGGGGPGGGGGPGGGGGSLYSDVSTGLTWRSGTASSVSYPKQMPLSQV",
# MLP3A_HUMAN
"MPSDRPFKQRRSFADRCKEVQQIRDQHPSKIPVIIERYKGEKQLPVLDKTKFLVPDHVNMSELVKIIRRRLQLNPTQAFFLLVNQHSMVSVSTPIADIYEQEKDEDGFLYMVYASQETFGF",
# WNK1_HUMAN
"MSGGAAEKQSSTPGSLFLSPPAPAPKNGSSSDSSVGEKLGAAAADAVTGRTEEYRRRRHTMDKDSRGAAATTTTTEHRFFRRSVICDSNATALELPGLPLSLPQPSIPAAVPQSAPPEPHREETVTATATSQVAQQPPAAAAPGEQAVAGPAPSTVPSSTSKDRPVSQPSLVGSKEEPPPARSGSGGGSAKEPQEERSQQQDDIEELETKAVGMSNDGRFLKFDIEIGRGSFKTVYKGLDTETTVEVAWCELQDRKLTKSERQRFKEEAEMLKGLQHPNIVRFYDSWESTVKGKKCIVLVTELMTSGTLKTYLKRFKVMKIKVLRSWCRQILKGLQFLHTRTPPIIHRDLKCDNIFITGPTGSVKIGDLGLATLKRASFAKSVIGTPEFMAPEMYEEKYDESVDVYAFGMCMLEMATSEYPYSECQNAAQIYRRVTSGVKPASFDKVAIPEVKEIIEGCIRQNKDERYSIKDLLNHAFFQEETGVRVELAEEDDGEKIAIKLWLRIEDIKKLKGKYKDNEAIEFSFDLERDVPEDVAQEMVESGYVCEGDHKTMAKAIKDRVSLIKRKREQRQLVREEQEKKKQEESSLKQQVEQSSASQTGIKQLPSASTGIPTASTTSASVSTQVEPEEPEADQHQQLQYQQPSISVLSDGTVDSGQGSSVFTESRVSSQQTVSYGSQHEQAHSTGTVPGHIPSTVQAQSQPHGVYPPSSVAQGQSQGQPSSSSLTGVSSSQPIQHPQQQQGIQQTAPPQQTVQYSLSQTSTSSEATTAQPVSQPQAPQVLPQVSAGKQLPVSQPVPTIQGEPQIPVATQPSVVPVHSGAHFLPVGQPLPTPLLPQYPVSQIPISTPHVSTAQTGFSSLPITMAAGITQPLLTLASSATTAAIPGVSTVVPSQLPTLLQPVTQLPSQVHPQLLQPAVQSMGIPANLGQAAEVPLSSGDVLYQGFPPRLPPQYPGDSNIAPSSNVASVCIHSTVLSPPMPTEVLATPGYFPTVVQPYVESNLLVPMGGVGGQVQVSQPGGSLAQAPTTSSQQAVLESTQGVSQVAPAEPVAVAQTQATQPTTLASSVDSAHSDVASGMSDGNENVPSSSGRHEGRTTKRHYRKSVRSRSRHEKTSRPKLRILNVSNKGDRVVECQLETHNRKMVTFKFDLDGDNPEEIATIMVNNDFILAIERESFVDQVREIIEKADEMLSEDVSVEPEGDQGLESLQGKDDYGFSGSQKLEGEFKQPIPASSMPQQIGIPTSSLTQVVHSAGRRFIVSPVPESRLRESKVFPSEITDTVAASTAQSPGMNLSHSASSLSLQQAFSELRRAQMTEGPNTAPPNFSHTGPTFPVVPPFLSSIAGVPTTAAATAPVPATSSPPNDISTSVIQSEVTVPTEEGIAGVATSTGVVTSGGLPIPPVSESPVLSSVVSSITIPAVVSISTTSPSLQVPTSTSEIVVSSTALYPSVTVSATSASAGGSTATPGPKPPAVVSQQAAGSTTVGATLTSVSTTTSFPSTASQLCIQLSSSTSTPTLAETVVVSAHSLDKTSHSSTTGLAFSLSAPSSSSSPGAGVSSYISQPGGLHPLVIPSVIASTPILPQAAGPTSTPLLPQVPSIPPLVQPVANVPAVQQTLIHSQPQPALLPNQPHTHCPEVDSDTQPKAPGIDDIKTLEEKLRSLFSEHSSSGAQHASVSLETSLVIESTVTPGIPTTAVAPSKLLTSTTSTCLPPTNLPLGTVALPVTPVVTPGQVSTPVSTTTSGVKPGTAPSKPPLTKAPVLPVGTELPAGTLPSEQLPPFPGPSLTQSQQPLEDLDAQLRRTLSPEMITVTSAVGPVSMAAPTAITEAGTQPQKGVSQVKEGPVLATSSGAGVFKMGRFQVSVAADGAQKEGKNKSEDAKSVHFESSTSESSVLSSSSPESTLVKPEPNGITIPGISSDVPESAHKTTASEAKSDTGQPTKVGRFQVTTTANKVGRFSVSKTEDKITDTKKEGPVASPPFMDLEQAVLPAVIPKKEKPELSEPSHLNGPSSDPEAAFLSRDVDDGSGSPHSPHQLSSKSLPSQNLSQSLSNSFNSSYMSSDNESDIEDEDLKLELRRLRDKHLKEIQDLQSRQKHEIESLYTKLGKVPPAVIIPPAAPLSGRRRRPTKSKGSKSSRSSSLGNKSPQLSGNLSGQSAASVLHPQQTLHPPGNIPESGQNQLLQPLKPSPSSDNLYSAFTSDGAISVPSLSAPGQGTSSTNTVGATVNSQAAQAQPPAMTSSRKGTFTDDLHKLVDNWARDAMNLSGRRGSKGHMNYEGPGMARKFSAPGQLCISMTSNLGGSAPISAASATSLGHFTKSMCPPQQYGFPATPFGAQWSGTGGPAPQPLGQFQPVGTASLQNFNISNLQKSISNPPGSNLRTT",
# CSTN2_HUMAN
"MLPGRLCWVPLLLALGVGSGSGGGGDSRQRRLLAAKVNKHKPWIETSYHGVITENNDTVILDPPLVALDKDAPVPFAGEICAFKIHGQELPFEAVVLNKTSGEGRLRAKSPIDCELQKEYTFIIQAYDCGAGPHETAWKKSHKAVVHIQVKDVNEFAPTFKEPAYKAVVTEGKIYDSILQVEAIDEDCSPQYSQICNYEIVTTDVPFAIDRNGNIRNTEKLSYDKQHQYEILVTAYDCGQKPAAQDTLVQVDVKPVCKPGWQDWTKRIEYQPGSGSMPLFPSIHLETCDGAVSSLQIVTELQTNYIGKGCDRETYSEKSLQKLCGASSGIIDLLPSPSAATNWTAGLLVDSSEMIFKFDGRQGAKVPDGIVPKNLTDQFTITMWMKHGPSPGVRAEKETILCNSDKTEMNRHHYALYVHNCRLVFLLRKDFDQADTFRPAEFHWKLDQICDKEWHYYVINVEFPVVTLYMDGATYEPYLVTNDWPIHPSHIAMQLTVGACWQGGEVTKPQFAQFFHGSLASLTIRPGKMESQKVISCLQACKEGLDINSLESLGQGIKYHFNPSQSILVMEGDDIGNINRALQKVSYINSRQFPTAGVRRLKVSSKVQCFGEDVCISIPEVDAYVMVLQAIEPRITLRGTDHFWRPAAQFESARGVTLFPDIKIVSTFAKTEAPGDVKTTDPKSEVLEEMLHNLDFCDILVIGGDLDPRQECLELNHSELHQRHLDATNSTAGYSIYGVGSMSRYEQVLHHIRYRNWRPASLEARRFRIKCSELNGRYTSNEFNLEVSILHEDQVSDKEHVNHLIVQPPFLQSVHHPESRSSIQHSSVVPSIATVVIIISVCMLVFVVAMGVYRVRIAHQHFIQETEAAKESEMDWDDSALTITVNPMEKHEGPGHGEDETEGEEEEEAEEEMSSSSGSDDSEEEEEEEGMGRGRHGQNGARQAQLEWDDSTLPY",
# E41L1_HUMAN
"MTTETGPDSEVKKAQEEAPQQPEAAAAVTTPVTPAGHGHPEANSNEKHPSQQDTRPAEQSLDMEEKDYSEADGLSERTTPSKAQKSPQKIAKKYKSAICRVTLLDASEYECEVEKHGRGQVLFDLVCEHLNLLEKDYFGLTFCDADSQKNWLDPSKEIKKQIRSSPWNFAFTVKFYPPDPAQLTEDITRYYLCLQLRADIITGRLPCSFVTHALLGSYAVQAELGDYDAEEHVGNYVSELRFAPNQTRELEERIMELHKTYRGMTPGEAEIHFLENAKKLSMYGVDLHHAKDSEGIDIMLGVCANGLLIYRDRLRINRFAWPKILKISYKRSNFYIKIRPGEYEQFESTIGFKLPNHRSAKRLWKVCIEHHTFFRLVSPEPPPKGFLVMGSKFRYSGRTQAQTRQASALIDRPAPFFERSSSKRYTMSRSLDGAEFSRPASVSENHDAGPDGDKRDEDGESGGQRSEAEEGEVRTPTKIKELKPEQETTPRHKQEFLDKPEDVLLKHQASINELKRTLKEPNSKLIHRDRDWERERRLPSSPASPSPKGTPEKANERAGLREGSEEKVKPPRPRAPESDTGDEDQDQERDTVFLKDNHLAIERKCSSITVSSTSSLEAEVDFTVIGDYHGSAFEDFSRSLPELDRDKSDSDTEGLLFSRDLNKGAPSQDDESGGIEDSPDRGACSTPDMPQFEPVKTETMTVSSLAIRKKIEPEAVLQTRVSAMDNTQQVDGSASVGREFIATTPSITTETISTTMENSLKSGKGAAAMIPGPQTVATEIRSLSPIIGKDVLTSTYGATAETLSTSTTTHVTKTVKGGFSETRIEKRIIITGDEDVDQDQALALAIKEAKLQHPDMLVTKAVVYRETDPSPEERDKKPQES",
# EHD1_HUMAN
"MFSWVSKDARRKKEPELFQTVAEGLRQLYAQKLLPLEEHYRFHEFHSPALEDADFDNKPMVLLVGQYSTGKTTFIRHLIEQDFPGMRIGPEPTTDSFIAVMHGPTEGVVPGNALVVDPRRPFRKLNAFGNAFLNRFMCAQLPNPVLDSISIIDTPGILSGEKQRISRGYDFAAVLEWFAERVDRIILLFDAHKLDISDEFSEVIKALKNHEDKIRVVLNKADQIETQQLMRVYGALMWSLGKIINTPEVVRVYIGSFWSHPLLIPDNRKLFEAEEQDLFKDIQSLPRNAALRKLNDLIKRARLAKVHAYIISSLKKEMPNVFGKESKKKELVNNLGEIYQKIEREHQISPGDFPSLRKMQELLQTQDFSKFQALKPKLLDTVDDMLANDIARLMVMVRQEESLMPSQVVKGGAFDGTMNGPFGHGYGEGAGEGIDDVEWVVGKDKPTYDEIFYTLSPVNGKITGANAKKEMVKSKLPNTVLGKIWKLADVDKDGLLDDEEFALANHLIKVKLEGHELPADLPPHLVPPSKRRHE",
# VIAAT_HUMAN
"MATLLRSKLSNVATSVSNKSQAKMSGMFARMGFQAATDEEAVGFAHCDDLDFEHRQGLQMDILKAEGEPCGDEGAEAPVEGDIHYQRGSGAPLPPSGSKDQVGGGGEFGGHDKPKITAWEAGWNVTNAIQGMFVLGLPYAILHGGYLGLFLIIFAAVVCCYTGKILIACLYEENEDGEVVRVRDSYVAIANACCAPRFPTLGGRVVNVAQIIELVMTCILYVVVSGNLMYNSFPGLPVSQKSWSIIATAVLLPCAFLKNLKAVSKFSLLCTLAHFVINILVIAYCLSRARDWAWEKVKFYIDVKKFPISIGIIVFSYTSQIFLPSLEGNMQQPSEFHCMMNWTHIAACVLKGLFALVAYLTWADETKEVITDNLPGSIRAVVNIFLVAKALLSYPLPFFAAVEVLEKSLFQEGSRAFFPACYSGDGRLKSWGLTLRCALVVFTLLMAIYVPHFALLMGLTGSLTGAGLCFLLPSLFHLRLLWRKLLWHQVFFDVAIFVIGGICSVSGFVHSLEGLIEAYRTNAED",
# CLMP_HUMAN
"MSLLLLLLLVSYYVGTLGTHTEIKRVAEEKVTLPCHHQLGLPEKDTLDIEWLLTDNEGNQKVVITYSSRHVYNNLTEEQKGRVAFASNFLAGDASLQIEPLKPSDEGRYTCKVKNSGRYVWSHVILKVLVRPSKPKCELEGELTEGSDLTLQCESSSGTEPIVYYWQRIREKEGEDERLPPKSRIDYNHPGRVLLQNLTMSYSGLYQCTAGNEAGKESCVVRVTVQYVQSIGMVAGAVTGIVAGALLIFLLVWLLIRRKDKERYEEEERPNEIREDAEAPKARLVKPSSSSSGSRSSRSGSSSTRSTANSASRSQRTLSTDAAPQPGLATQAYSLVGPEVRGSEPKKVHHANLTKAETTPSMIPSQSRAFQTV",
# ANTR1_HUMAN
"MATAERRALGIGFQWLSLATLVLICAGQGGRREDGGPACYGGFDLYFILDKSGSVLHHWNEIYYFVEQLAHKFISPQLRMSFIVFSTRGTTLMKLTEDREQIRQGLEELQKVLPGGDTYMHEGFERASEQIYYENRQGYRTASVIIALTDGELHEDLFFYSEREANRSRDLGAIVYCVGVKDFNETQLARIADSKDHVFPVNDGFQALQGIIHSILKKSCIEILAAEPSTICAGESFQVVVRGNGFRHARNVDRVLCSFKINDSVTLNEKPFSVEDTYLLCPAPILKEVGMKAALQVSMNDGLSFISSSVIITTTHCSDGSILAIALLILFLLLALALLWWFWPLCCTVIIKEVPPPPAEESEEEDDDGLPKKKWPTVDASYYGGRGVGGIKRMEVRWGEKGSTEEGAKLEKAKNARVKMPEQEYEFPEPRNLNNNMRRPSSPRKWYSPIKGKLDALWVLLRKGYDRVSVMRPQPGDTGRCINFTRVKNNQPAKYPLNNAYHTSSPPPAPIYTPPPPAPHCPPPPPSAPTPPIPSPPSTLPPPPQAPPPNRAPPPSRPPPRPSV",
# SYNG1_HUMAN
"MDGIIEQKSMLVHSKISDAGKRNGLINTRNLMAESRDGLVSVYPAPQYQSHRVGASTVPASLDSSRSEPMQQLLDPNTLQQSVESRYRPNIILYSEGVLRSWGDGVAADCCETTFIEDRSPTKDSLEYPDGKFIDLSADDIKIHTLSYDVEEEEEFQELESDYSSDTESEDNFLMMPPRDHLGLSVFSMLCCFWPLGIAAFYLSHETNKAVAKGDLHQASTSSRRALFLAVLSITIGTGVYVGVAVALIAYLSKNNHL",
# CNNM2_HUMAN
"MIGCGACEPKVKMAGGQAAAALPTWKMAARRSLSARGRGILQAAAGRLLPLLLLSCCCGAGGCAAVGENEETVIIGLRLEDTNDVSFMEGGALRVSERTRVKLRVYGQNINNETWSRIAFTEHERRRHSPGERGLGGPAPPEPDSGPQRCGIRTSDIIILPHIILNRRTSGIIEIEIKPLRKMEKSKSYYLCTSLSTPALGAGGSGSTGGAVGGKGGSGVAGLPPPPWAETTWIYHDGEDTKMIVGEEKKFLLPFWLQVIFISLLLCLSGMFSGLNLGLMALDPMELRIVQNCGTEKEKNYAKRIEPVRRQGNYLLCSLLLGNVLVNTTLTILLDDIAGSGLVAVVVSTIGIVIFGEIVPQAICSRHGLAVGANTIFLTKFFMMMTFPASYPVSKLLDCVLGQEIGTVYNREKLLEMLRVTDPYNDLVKEELNIIQGALELRTKTVEDVMTPLRDCFMITGEAILDFNTMSEIMESGYTRIPVFEGERSNIVDLLFVKDLAFVDPDDCTPLKTITKFYNHPLHFVFNDTKLDAMLEEFKKGKSHLAIVQRVNNEGEGDPFYEVLGIVTLEDVIEEIIKSEILDETDLYTDNRTKKKVAHRERKQDFSAFKQTDSEMKVKISPQLLLAMHRFLATEVEAFSPSQMSEKILLRLLKHPNVIQELKYDEKNKKAPEYYLYQRNKPVDYFVLILQGKVEVEAGKEGMKFEASAFSYYGVMALTASPVPLSLSRTFVVSRTELLAAGSPGENKSPPRPCGLNHSDSLSRSDRIDAVTPTLGSSNNQLNSSLLQVYIPDYSVRALSDLQFVKISRQQYQNALMASRMDKTPQSSDSENTKIELTLTELHDGLPDETANLLNEQNCVTHSKANHSLHNEGAI",
# LIN7B_HUMAN
"MAALVEPLGLERDVSRAVELLERLQRSGELPPQKLQALQRVLQSRFCSAIREVYEQLYDTLDITGSAEIRAHATAKATVAAFTASEGHAHPRVVELPKTDEGLGFNIMGGKEQNSPIYISRVIPGGVADRHGGLKRGDQLLSVNGVSVEGEQHEKAVELLKAAQGSVKLVVRYTPRVLEEMEARFEKMRSARRRQQHQSYSSLESRG",
# AGRL3_HUMAN
"MWPSQLLIFMMLLAPIIHAFSRAPIPMAVVRRELSCESYPIELRCPGTDVIMIESANYGRTDDKICDSDPAQMENIRCYLPDAYKIMSQRCNNRTQCAVVAGPDVFPDPCPGTYKYLEVQYECVPYKVEQKVFLCPGLLKGVYQSEHLFESDHQSGAWCKDPLQASDKIYYMPWTPYRTDTLTEYSSKDDFIAGRPTTTYKLPHRVDGTGFVVYDGALFFNKERTRNIVKFDLRTRIKSGEAIIANANYHDTSPYRWGGKSDIDLAVDENGLWVIYATEQNNGKIVISQLNPYTLRIEGTWDTAYDKRSASNAFMICGILYVVKSVYEDDDNEATGNKIDYIYNTDQSKDSLVDVPFPNSYQYIAAVDYNPRDNLLYVWNNYHVVKYSLDFGPLDSRSGQAHHGQVSYISPPIHLDSELERPSVKDISTTGPLGMGSTTTSTTLRTTTLSPGRSTTPSVSGRRNRSTSTPSPAVEVLDDMTTHLPSASSQIPALEESCEAVEAREIMWFKTRQGQIAKQPCPAGTIGVSTYLCLAPDGIWDPQGPDLSNCSSPWVNHITQKLKSGETAANIARELAEQTRNHLNAGDITYSVRAMDQLVGLLDVQLRNLTPGGKDSAARSLNKAMVETVNNLLQPQALNAWRDLTTSDQLRAATMLLHTVEESAFVLADNLLKTDIVRENTDNIKLEVARLSTEGNLEDLKFPENMGHGSTIQLSANTLKQNGRNGEIRVAFVLYNNLGPYLSTENASMKLGTEALSTNHSVIVNSPVITAAINKEFSNKVYLADPVVFTVKHIKQSEENFNPNCSFWSYSKRTMTGYWSTQGCRLLTTNKTHTTCSCNHLTNFAVLMAHVEVKHSDAVHDLLLDVITWVGILLSLVCLLICIFTFCFFRGLQSDRNTIHKNLCISLFVAELLFLIGINRTDQPIACAVFAALLHFFFLAAFTWMFLEGVQLYIMLVEVFESEHSRRKYFYLVGYGMPALIVAVSAAVDYRSYGTDKVCWLRLDTYFIWSFIGPATLIIMLNVIFLGIALYKMFHHTAILKPESGCLDNIKSWVIGAIALLCLLGLTWAFGLMYINESTVIMAYLFTIFNSLQGMFIFIFHCVLQKKVRKEYGKCLRTHCCSGKSTESSIGSGKTSGSRTPGRYSTGSQSRIRRMWNDTVRKQSESSFITGDINSSASLNREGLLNNARDTSVMDTLPLNGNHGNSYSIASGEYLSNCVQIIDRGYNHNETALEKKILKELTSNYIPSYLNNHERSSEQNRNLMNKLVNNLGSGREDDAIVLDDATSFNHEESLGLELIHEESDAPLLPPRVYSTENHQPHHYTRRRIPQDHSESFFPLLTNEHTEDLQSPHRDSLYTSMPTLAGVAATESVTTSTQTEPPPAKCGDAEDVYYKSMPNLGSRNHVHQLHTYYQLGRGSSDGFIVPPNKDGTPPEGSSKGPAHLVTSL",
# PKHA5_HUMAN
"MAADLNLEWISLPRSWTYGITRGGRVFFINEEAKSTTWLHPVTGEAVVTGHRRQSTDLPTGWEEAYTFEGARYYINHNERKVTCKHPVTGQPSQDNCIFVVNEQTVATMTSEEKKERPISMINEASNYNVTSDYAVHPMSPVGRTSRASKKVHNFGKRSNSIKRNPNAPVVRRGWLYKQDSTGMKLWKKRWFVLSDLCLFYYRDEKEEGILGSILLPSFQIALLTSEDHINRKYAFKAAHPNMRTYYFCTDTGKEMELWMKAMLDAALVQTEPVKRVDKITSENAPTKETNNIPNHRVLIKPEIQNNQKNKEMSKIEEKKALEAEKYGFQKDGQDRPLTKINSVKLNSLPSEYESGSACPAQTVHYRPINLSSSENKIVNVSLADLRGGNRPNTGPLYTEADRVIQRTNSMQQLEQWIKIQKGRGHEEETRGVISYQTLPRNMPSHRAQIMARYPEGYRTLPRNSKTRPESICSVTPSTHDKTLGPGAEEKRRSMRDDTMWQLYEWQQRQFYNKQSTLPRHSTLSSPKTMVNISDQTMHSIPTSPSHGSIAAYQGYSPQRTYRSEVSSPIQRGDVTIDRRHRAHHPKHVYVPDRRSVPAGLTLQSVSPQSLQGKTLSQDEGRGTLYKYRPEEVDIDAKLSRLCEQDKVVHALEEKLQQLHKEKYTLEQALLSASQEIEMHADNPAAIQTVVLQRDDLQNGLLSTCRELSRATAELERAWREYDKLEYDVTVTRNQMQEQLDHLGEVQTESAGIQRAQIQKELWRIQDVMEGLSKHKQQRGTTEIGMIGSKPFSTVKYKNEGPDYRLYKSEPELTTVAEVDESNGEEKSEPVSEIETSVVKGSHFPVGVVPPRAKSPTPESSTIASYVTLRKTKKMMDLRTERPRSAVEQLCLAESTRPRMTVEEQMERIRRHQQACLREKKKGLNVIGASDQSPLQSPSNLRDNPFRTTQTRRRDDKELDTAIRENDVKPDHETPATEIVQLKETEPQNVDFSKELKKTENISYEMLFEPEPNGVNSVEMMDKERNKDKMPEDVTFSPQDETQTANHKPEEHPEENTKNSVDEQEETVISYESTPEVSRGNQTMAVKSLSPSPESSASPVPSTQPQLTEGSHFMCV",
# GBB4_HUMAN
"MSELEQLRQEAEQLRNQIQDARKACNDATLVQITSNMDSVGRIQMRTRRTLRGHLAKIYAMHWGYDSRLLVSASQDGKLIIWDSYTTNKMHAIPLRSSWVMTCAYAPSGNYVACGGLDNICSIYNLKTREGNVRVSRELPGHTGYLSCCRFLDDSQIVTSSGDTTCALWDIETAQQTTTFTGHSGDVMSLSLSPDMRTFVSGACDASSKLWDIRDGMCRQSFTGHVSDINAVSFFPNGYAFATGSDDATCRLFDLRADQELLLYSHDNIICGITSVAFSKSGRLLLAGYDDFNCNVWDTLKGDRAGVLAGHDNRVSCLGVTDDGMAVATGSWDSFLRIWN",
# VPP4_HUMAN
"MVSVFRSEEMCLSQLFLQVEAAYCCVAELGELGLVQFKDLNMNVNSFQRKFVNEVRRCESLERILRFLEDEMQNEIVVQLLEKSPLTPLPREMITLETVLEKLEGELQEANQNQQALKQSFLELTELKYLLKKTQDFFETETNLADDFFTEDTSGLLELKAVPAYMTGKLGFIAGVINRERMASFERLLWRICRGNVYLKFSEMDAPLEDPVTKEEIQKNIFIIFYQGEQLRQKIKKICDGFRATVYPCPEPAVERREMLESVNVRLEDLITVITQTESHRQRLLQEAAANWHSWLIKVQKMKAVYHILNMCNIDVTQQCVIAEIWFPVADATRIKRALEQGMELSGSSMAPIMTTVQSKTAPPTFNRTNKFTAGFQNIVDAYGVGSYREINPAPYTIITFPFLFAVMFGDCGHGTVMLLAALWMILNERRLLSQKTDNEIWNTFFHGRYLILLMGIFSIYTGLIYNDCFSKSLNIFGSSWSVQPMFRNGTWNTHVMEESLYLQLDPAIPGVYFGNPYPFGIDPIWNLASNKLTFLNSYKMKMSVILGIVQMVFGVILSLFNHIYFRRTLNIILQFIPEMIFILCLFGYLVFMIIFKWCCFDVHVSQHAPSILIHFINMFLFNYSDSSNAPLYKHQQEVQSFFVVMALISVPWMLLIKPFILRASHRKSQLQASRIQEDATENIEGDSSSPSSRSGQRTSADTHGALDDHGEEFNFGDVFVHQAIHTIEYCLGCISNTASYLRLWALSLAHAQLSEVLWTMVMNSGLQTRGWGGIVGVFIIFAVFAVLTVAILLIMEGLSAFLHALRLHWVEFQNKFYVGDGYKFSPFSFKHILDGTAEE",
# VEZA_HUMAN
"MTPEFDEEVVFENSPLYQYLQDLGHTDFEICSSLSPKTEKCTTEGQQKPPTRVLPKQGILLKVAETIKSWIFFSQCNKKDDLLHKLDIGFRLDSLHTILQQEVLLQEDVELIELLDPSILSAGQSQQQENGHLPTLCSLATPNIWDLSMLFAFISLLVMLPTWWIVSSWLVWGVILFVYLVIRALRLWRTAKLQVTLKKYSVHLEDMATNSRAFTNLVRKALRLIQETEVISRGFTLVSAACPFNKAGQHPSQHLIGLRKAVYRTLRANFQAARLATLYMLKNYPLNSESDNVTNYICVVPFKELGLGLSEEQISEEEAHNFTDGFSLPALKVLFQLWVAQSSEFFRRLALLLSTANSPPGPLLTPALLPHRILSDVTQGLPHAHSACLEELKRSYEFYRYFETQHQSVPQCLSKTQQKSRELNNVHTAVRSLQLHLKALLNEVIILEDELEKLVCTKETQELVSEAYPILEQKLKLIQPHVQASNNCWEEAISQVDKLLRRNTDKKGKPEIACENPHCTVVPLKQPTLHIADKDPIPEEQELEAYVDDIDIDSDFRKDDFYYLSQEDKERQKREHEESKRVLQELKSVLGFKASEAERQKWKQLLFSDHAVLKSLSPVDPVEPISNSEPSMNSDMGKVSKNDTEEESNKSATTDNEISRTEYLCENSLEGKNKDNSSNEVFPQGAEERMCYQCESEDEPQADGSGLTTAPPTPRDSLQPSIKQRLARLQLSPDFTFTAGLAAEVAARSLSFTTMQEQTFGGEEEEQIIEENKNEIEEK",
# LPAR2_HUMAN
"MGQCYYNETIGFFYNNSGKELSSHWRPKDVVVVALGLTVSVLVLLTNLLVIAAIASNRRFHQPIYYLLGNLAAADLFAGVAYLFLMFHTGPRTARLSLEGWFLRQGLLDTSLTASVATLLAIAVERHRSVMAVQLHSRLPRGRVVMLIVGVWVAALGLGLLPAHSWHCLCALDRCSRMAPLLSRSYLAVWALSSLLVFLLMVAVYTRIFFYVRRRVQRMAEHVSCHPRYRETTLSLVKTVVIILGAFVVCWTPGQVVLLLDGLGCESCNVLAVEKYFLLLAEANSLVNAAVYSCRDAEMRRTFRRLLCCACLRQSTRESVHYTSSAQGGASTRIMLPENGHPLMDSTL",
# LRRC4_HUMAN
"MKLLWQVTVHHHTWNAILLPFVYLTAQVWILCAAIAAAASAGPQNCPSVCSCSNQFSKVVCTRRGLSEVPQGIPSNTRYLNLMENNIQMIQADTFRHLHHLEVLQLGRNSIRQIEVGAFNGLASLNTLELFDNWLTVIPSGAFEYLSKLRELWLRNNPIESIPSYAFNRVPSLMRLDLGELKKLEYISEGAFEGLFNLKYLNLGMCNIKDMPNLTPLVGLEELEMSGNHFPEIRPGSFHGLSSLKKLWVMNSQVSLIERNAFDGLASLVELNLAHNNLSSLPHDLFTPLRYLVELHLHHNPWNCDCDILWLAWWLREYIPTNSTCCGRCHAPMHMRGRYLVEVDQASFQCSAPFIMDAPRDLNISEGRMAELKCRTPPMSSVKWLLPNGTVLSHASRHPRISVLNDGTLNFSHVLLSDTGVYTCMVTNVAGNSNASAYLNVSTAELNTSNYSFFTTVTVETTEISPEDTTRKYKPVPTTSTGYQPAYTTSTTVLIQTTRVPKQVAVPATDTTDKMQTSLDEVMKTTKIIIGCFVAVTLLAAAMLIVFYKLRKRHQQRSTVTAARTVEIIQVDEDIPAATSAAATAAPSGVSGEGAVVLPTIHDHINYNTYKPAHGAHWTENSLGNSLHPTVTTISEPYIIQTHTKDKVQETQI",
# OTOF_HUMAN
"MALLIHLKTVSELRGRGDRIAKVTFRGQSFYSRVLENCEDVADFDETFRWPVASSIDRNEMLEIQVFNYSKVFSNKLIGTFRMVLQKVVEESHVEVTDTLIDDNNAIIKTSLCVEVRYQATDGTVGSWDDGDFLGDESLQEEEKDSQETDGLLPGSRPSSRPPGEKSFRRAGRSVFSAMKLGKNRSHKEEPQRPDEPAVLEMEDLDHLAIRLGDGLDPDSVSLASVTALTTNVSNKRSKPDIKMEPSAGRPMDYQVSITVIEARQLVGLNMDPVVCVEVGDDKKYTSMKESTNCPYYNEYFVFDFHVSPDVMFDKIIKISVIHSKNLLRSGTLVGSFKMDVGTVYSQPEHQFHHKWAILSDPDDISSGLKGYVKCDVAVVGKGDNIKTPHKANETDEDDIEGNLLLPEGVPPERQWARFYVKIYRAEGLPRMNTSLMANVKKAFIGENKDLVDPYVQVFFAGQKGKTSVQKSSYEPLWNEQVVFTDLFPPLCKRMKVQIRDSDKVNDVAIGTHFIDLRKISNDGDKGFLPTLGPAWVNMYGSTRNYTLLDEHQDLNEGLGEGVSFRARLLLGLAVEIVDTSNPELTSSTEVQVEQATPISESCAGKMEEFFLFGAFLEASMIDRRNGDKPITFEVTIGNYGNEVDGLSRPQRPRPRKEPGDEEEVDLIQNASDDEAGDAGDLASVSSTPPMRPQVTDRNYFHLPYLERKPCIYIKSWWPDQRRRLYNANIMDHIADKLEEGLNDIQEMIKTEKSYPERRLRGVLEELSCGCCRFLSLADKDQGHSSRTRLDRERLKSCMRELENMGQQARMLRAQVKRHTVRDKLRLCQNFLQKLRFLADEPQHSIPDIFIWMMSNNKRVAYARVPSKDLLFSIVEEETGKDCAKVKTLFLKLPGKRGFGSAGWTVQAKVELYLWLGLSKQRKEFLCGLPCGFQEVKAAQGLGLHAFPPVSLVYTKKQAFQLRAHMYQARSLFAADSSGLSDPFARVFFINQSQCTEVLNETLCPTWDQMLVFDNLELYGEAHELRDDPPIIVIEIYDQDSMGKADFMGRTFAKPLVKMADEAYCPPRFPPQLEYYQIYRGNATAGDLLAAFELLQIGPAGKADLPPINGPVDVDRGPIMPVPMGIRPVLSKYRVEVLFWGLRDLKRVNLAQVDRPRVDIECAGKGVQSSLIHNYKKNPNFNTLVKWFEVDLPENELLHPPLNIRVVDCRAFGRYTLVGSHAVSSLRRFIYRPPDRSAPSWNTTVRLLRRCRVLCNGGSSSHSTGEVVVTMEPEVPIKKLETMVKLDATSEAVVKVDVAEEEKEKKKKKKGTAEEPEEEEPDESMLDWWSKYFASIDTMKEQLRQQEPSGIDLEEKEEVDNTEGLKGSMKGKEKARAAKEEKKKKTQSSGSGQGSEAPEKKKPKIDELKVYPKELESEFDNFEDWLHTFNLLRGKTGDDEDGSTEEERIVGRFKGSLCVYKVPLPEDVSREAGYDSTYGMFQGIPSNDPINVLVRVYVVRATDLHPADINGKADPYIAIRLGKTDIRDKENYISKQLNPVFGKSFDIEASFPMESMLTVAVYDWDLVGTDDLIGETKIDLENRFYSKHRATCGIAQTYSTHGYNIWRDPMKPSQILTRLCKDGKVDGPHFGPPGRVKVANRVFTGPSEIEDENGQRKPTDEHVALLALRHWEDIPRAGCRLVPEHVETRPLLNPDKPGIEQGRLELWVDMFPMDMPAPGTPLDISPRKPKKYELRVIIWNTDEVVLEDDDFFTGEKSSDIFVRGWLKGQQEDKQDTDVHYHSLTGEGNFNWRYLFPFDYLAAEEKIVISKKESMFSWDETEYKIPARLTLQIWDADHFSADDFLGAIELDLNRFPRGAKTAKQCTMEMATGEVDVPLVSIFKQKRVKGWWPLLARNENDEFELTGKVEAELHLLTAEEAEKNPVGLARNEPDPLEKPNRPDTSFIWFLNPLKSARYFLWHTYRWLLLKLLLLLLLLLLLALFLYSVPGYLVKKILGA",
# PCDH9_HUMAN
"MDLRDFYLLAALIACLRLDSAIAQELIYTIREELPENVPIGNIPKDLNISHINAATGTSASLVYRLVSKAGDAPLVKVSSSTGEIFTTSNRIDREKLCAGASYAEENECFFELEVVILPNDFFRLIKIKIIVKDTNDNAPMFPSPVINISIPENTLINSRFPIPSATDPDTGFNGVQHYELLNGQSVFGLDIVETPEGEKWPQLIVQQNLDREQKDTYVMKIKVEDGGTPQKSSTAILQVTVSDVNDNRPVFKEGQVEVHIPENAPVGTSVIQLHATDADIGSNAEIRYIFGAQVAPATKRLFALNNTTGLITVQRSLDREETAIHKVTVLASDGSSTPARATVTINVTDVNDNPPNIDLRYIISPINGTVYLSEKDPVNTKIALITVSDKDTDVNGKVICFIEREVPFHLKAVYDNQYLLETSSLLDYEGTKEFSFKIVASDSGKPSLNQTALVRVKLEDENDNPPIFNQPVIELSVSENNRRGLYLTTISATDEDSGKNADIVYQLGPNASFFDLDRKTGVLTASRVFDREEQERFIFTVTARDNGTPPLQSQAAVIVTVLDENDNSPKFTHNHFQFFVSENLPKYSTVGVITVTDADAGENKAVTLSILNDNDNFVLDPYSGVIKSNVSFDREQQSSYTFDVKATDGGQPPRSSTAKVTINVMDVNDNSPVVISPPSNTSFKLVPLSAIPGSVVAEVFAVDVDTGMNAELKYTIVSGNNKGLFRIDPVTGNITLEEKPAPTDVGLHRLVVNISDLGYPKSLHTLVLVFLYVNDTAGNASYIYDLIRRTMETPLDRNIGDSSQPYQNEDYLTIMIAIIAGAMVVIVVIFVTVLVRCRHASRFKAAQRSKQGAEWMSPNQENKQNKKKKRKKRKSPKSSLLNFVTIEESKPDDAVHEPINGTISLPAELEEQSIGRFDWGPAPPTTFKPNSPDLAKHYKSASPQPAFHLKPDTPVSVKKHHVIQELPLDNTFVGGCDTLSKRSSTSSDHFSASECSSQGGFKTKGPLHTRQCNSHSKSDNIPVTPQKCPSSTGFHIQENEESHYESQRRVTFHLPDGSQESCSDSGLGDHEPVGSGTLISHPLPLVQPQDEFYDQASPDKRTEADGNSDPNSDGPLGPRGLAEATEMCTQECLVLGHSDNCWMPPGLGPYQHPKSPLSTFAPQKEWVKKDKLVNGHTLTRAWKEDSNRNQFNDRKQYGSNEGHFNNGSHMTDIPLANLKSYKQAGGATESPKEHQL",
# TANC2_HUMAN
"MFRNSLKMLLTGGKSSRKNRSSDGGSEEPPDRRQSSVDSRQSRSGQGGISTESDCAFEPDYAVPPLPVSEGDAEQELGPPPSVDEAANTLMTRLGFLLGEKVTEVQPGDQYSMEVQDENQTSAITQRISPCSTLTSSTASPPASSPCSTLPPISTNATAKDCSYGAVTSPTSTLESRDSGIIATLTSYSENVERTKYAGESSKELGSGGNIKPWQSQKSSMDSCLYRVDENMTASTYSLNKIPERNLETVLSQSVQSIPLYLMPRPNSVAATSSAHLEDLAYLDEQRHTPLRTSLRMPRQSMGGARTQQDLRVRFAPYRPPDISLKPLLFEVPSITTESVFVGRDWVFHEIDAQLQSSNASVNQGVVIVGNIGFGKTAIISRLVALSCHGTRMRQIASDSPHASPKHVDANRELPLTQPPSAHSSITSGSCPGTPEMRRRQEEAMRRLASQVVAYHYCQADNAYTCLVPEFVHNVAALLCRSPQLTAYREQLLREPHLQSMLSLRSCVQDPMASFRRGVLEPLENLHKERKIPDEDFIILIDGLNEAEFHKPDYGDTIVSFLSKMIGKFPSWLKLIVTVRTSLQEITKLLPFHRIFLDRLEENEAIDQDLQAYILHRIHSSSEIQNNISLNGKMDNTTFGKLSSHLKTLSQGSYLYLKLTFDLIEKGYLVLKSSSYKVVPVSLSEVYLLQCNMKFPTQSSFDRVMPLLNVAVASLHPLTDEHIFQAINAGSIEGTLEWEDFQQRMENLSMFLIKRRDMTRMFVHPSFREWLIWREEGEKTKFLCDPRSGHTLLAFWFSRQEGKLNRQQTIELGHHILKAHIFKGLSKKVGVSSSILQGLWISYSTEGLSMALASLRNLYTPNIKVSRLLILGGANINYRTEVLNNAPILCVQSHLGYTEMVALLLEFGANVDASSESGLTPLGYAAAAGYLSIVVLLCKKRAKVDHLDKNGQCALVHAALRGHLEVVKFLIQCDWTMAGQQQGVFKKSHAIQQALIAAASMGYTEIVSYLLDLPEKDEEEVERAQINSFDSLWGETALTAAAGRGKLEVCRLLLEQGAAVAQPNRRGAVPLFSTVRQGHWQIVDLLLTHGADVNMADKQGRTPLMMAASEGHLGTVDFLLAQGASIALMDKEGLTALSWACLKGHLSVVRSLVDNGAATDHADKNGRTPLDLAAFYGDAEVVQFLVDHGAMIEHVDYSGMRPLDRAVGCRNTSVVVTLLKKGAKIGPATWAMATSKPDIMIILLSKLMEEGDMFYKKGKVKEAAQRYQYALKKFPREGFGEDLKTFRELKVSLLLNLSRCRRKMNDFGMAEEFATKALELKPKSYEAYYARARAKRSSRQFAAALEDLNEAIKLCPNNREIQRLLLRVEEECRQMQQPQQPPPPPQPQQQLPEEAEPEPQHEDIYSVQDIFEEEYLEQDVENVSIGLQTEARPSQGLPVIQSPPSSPPHRDSAYISSSPLGSHQVFDFRSSSSVGSPTRQTYQSTSPALSPTHQNSHYRPSPPHTSPAHQGGSYRFSPPPVGGQGKEYPSPPPSPLRRGPQYRASPPAESMSVYRSQSGSPVRYQQETSVSQLPGRPKSPLSKMAQRPYQMPQLPVAVPQQGLRLQPAKAQIVRSNQPSPAVHSSTVIPTGAYGQVAHSMASKYQSSQGDIGVSQSRLVYQGSIGGIVGDGRPVQHVQASLSAGAICQHGGLTKEDLPQRPSSAYRGGVRYSQTPQIGRSQSASYYPVCHSKLDLERSSSQLGSPDVSHLIRRPISVNPNEIKPHPPTPRPLLHSQSVGLRFSPSSNSISSTSNLTPTFRPSSSIQQMEIPLKPAYERSCDELSPVSPTQGGYPSEPTRSRTTPFMGIIDKTARTQQYPHLHQQNRTWAVSSVDTVLSPTSPGNLPQPESFSPPSSISNIAFYNKTNNAQNGHLLEDDYYSPHGMLANGSRGDLLERVSQASSYPDVKVARTLPVAQAYQDNLYRQLSRDSRQGQTSPIKPKRPFVESNV",
# LRC4C_HUMAN
"MLNKMTLHPQQIMIGPRFNRALFDPLLVVLLALQLLVVAGLVRAQTCPSVCSCSNQFSKVICVRKNLREVPDGISTNTRLLNLHENQIQIIKVNSFKHLRHLEILQLSRNHIRTIEIGAFNGLANLNTLELFDNRLTTIPNGAFVYLSKLKELWLRNNPIESIPSYAFNRIPSLRRLDLGELKRLSYISEGAFEGLSNLRYLNLAMCNLREIPNLTPLIKLDELDLSGNHLSAIRPGSFQGLMHLQKLWMIQSQIQVIERNAFDNLQSLVEINLAHNNLTLLPHDLFTPLHHLERIHLHHNPWNCNCDILWLSWWIKDMAPSNTACCARCNTPPNLKGRYIGELDQNYFTCYAPVIVEPPADLNVTEGMAAELKCRASTSLTSVSWITPNGTVMTHGAYKVRIAVLSDGTLNFTNVTVQDTGMYTCMVSNSVGNTTASATLNVTAATTTPFSYFSTVTVETMEPSQDEARTTDNNVGPTPVVDWETTNVTTSLTPQSTRSTEKTFTIPVTDINSGIPGIDEVMKTTKIIIGCFVAITLMAAVMLVIFYKMRKQHHRQNHHAPTRTVEIINVDDEITGDTPMESHLPMPAIEHEHLNHYNSYKSPFNHTTTVNTINSIHSSVHEPLLIRMNSKDNVQETQI",
# ROBO2_HUMAN
"MSLLMFTQLLLCGFLYVRVDGSRLRQEDFPPRIVEHPSDVIVSKGEPTTLNCKAEGRPTPTIEWYKDGERVETDKDDPRSHRMLLPSGSLFFLRIVHGRRSKPDEGSYVCVARNYLGEAVSRNASLEVALLRDDFRQNPTDVVVAAGEPAILECQPPRGHPEPTIYWKKDKVRIDDKEERISIRGGKLMISNTRKSDAGMYTCVGTNMVGERDSDPAELTVFERPTFLRRPINQVVLEEEAVEFRCQVQGDPQPTVRWKKDDADLPRGRYDIKDDYTLRIKKTMSTDEGTYMCIAENRVGKMEASATLTVRAPPQFVVRPRDQIVAQGRTVTFPCETKGNPQPAVFWQKEGSQNLLFPNQPQQPNSRCSVSPTGDLTITNIQRSDAGYYICQALTVAGSILAKAQLEVTDVLTDRPPPIILQGPANQTLAVDGTALLKCKATGDPLPVISWLKEGFTFPGRDPRATIQEQGTLQIKNLRISDTGTYTCVATSSSGETSWSAVLDVTESGATISKNYDLSDLPGPPSKPQVTDVTKNSVTLSWQPGTPGTLPASAYIIEAFSQSVSNSWQTVANHVKTTLYTVRGLRPNTIYLFMVRAINPQGLSDPSPMSDPVRTQDISPPAQGVDHRQVQKELGDVLVRLHNPVVLTPTTVQVTWTVDRQPQFIQGYRVMYRQTSGLQATSSWQNLDAKVPTERSAVLVNLKKGVTYEIKVRPYFNEFQGMDSESKTVRTTEEAPSAPPQSVTVLTVGSYNSTSISVSWDPPPPDHQNGIIQEYKIWCLGNETRFHINKTVDAAIRSVIIGGLFPGIQYRVEVAASTSAGVGVKSEPQPIIIGRRNEVVITENNNSITEQITDVVKQPAFIAGIGGACWVILMGFSIWLYWRRKKRKGLSNYAVTFQRGDGGLMSNGSRPGLLNAGDPSYPWLADSWPATSLPVNNSNSGPNEIGNFGRGDVLPPVPGQGDKTATMLSDGAIYSSIDFTTKTSYNSSSQITQATPYATTQILHSNSIHELAVDLPDPQWKSSIQQKTDLMGFGYSLPDQNKGNNGGKGGKKKKNKNSSKPQKNNGSTWANVPLPPPPVQPLPGTELEHYAVEQQENGYDSDSWCPPLPVQTYLHQGLEDELEEDDDRVPTPPVRGVASSPAISFGQQSTATLTPSPREEMQPMLQAHLDELTRAYQFDIAKQTWHIQSNNQPPQPPVPPLGYVSGALISDLETDVADDDADDEEEALEIPRPLRALDQTPGSSMDNLDSSVTGKAFTSSQRPRPTSPFSTDSNTSAALSQSQRPRPTKKHKGGRMDQQPALPHRREGMTDEEALVPYSKPSFPSPGGHSSSGTASSKGSTGPRKTEVLRAGHQRNASDLLDIGYMGSNSQGQFTGEL",
# PLXA4_HUMAN
"MKAMPWNWTCLLSHLLMVGMGSSTLLTRQPAPLSQKQRSFVTFRGEPAEGFNHLVVDERTGHIYLGAVNRIYKLSSDLKVLVTHETGPDEDNPKCYPPRIVQTCNEPLTTTNNVNKMLLIDYKENRLIACGSLYQGICKLLRLEDLFKLGEPYHKKEHYLSGVNESGSVFGVIVSYSNLDDKLFIATAVDGKPEYFPTISSRKLTKNSEADGMFAYVFHDEFVASMIKIPSDTFTIIPDFDIYYVYGFSSGNFVYFLTLQPEMVSPPGSTTKEQVYTSKLVRLCKEDTAFNSYVEVPIGCERSGVEYRLLQAAYLSKAGAVLGRTLGVHPDDDLLFTVFSKGQKRKMKSLDESALCIFILKQINDRIKERLQSCYRGEGTLDLAWLKVKDIPCSSALLTIDDNFCGLDMNAPLGVSDMVRGIPVFTEDRDRMTSVIAYVYKNHSLAFVGTKSGKLKKIRVDGPRGNALQYETVQVVDPGPVLRDMAFSKDHEQLYIMSERQLTRVPVESCGQYQSCGECLGSGDPHCGWCVLHNTCTRKERCERSKEPRRFASEMKQCVRLTVHPNNISVSQYNVLLVLETYNVPELSAGVNCTFEDLSEMDGLVVGNQIQCYSPAAKEVPRIITENGDHHVVQLQLKSKETGMTFASTSFVFYNCSVHNSCLSCVESPYRCHWCKYRHVCTHDPKTCSFQEGRVKLPEDCPQLLRVDKILVPVEVIKPITLKAKNLPQPQSGQRGYECILNIQGSEQRVPALRFNSSSVQCQNTSYSYEGMEINNLPVELTVVWNGHFNIDNPAQNKVHLYKCGAMRESCGLCLKADPDFACGWCQGPGQCTLRQHCPAQESQWLELSGAKSKCTNPRITEIIPVTGPREGGTKVTIRGENLGLEFRDIASHVKVAGVECSPLVDGYIPAEQIVCEMGEAKPSQHAGFVEICVAVCRPEFMARSSQLYYFMTLTLSDLKPSRGPMSGGTQVTITGTNLNAGSNVVVMFGKQPCLFHRRSPSYIVCNTTSSDEVLEMKVSVQVDRAKIHQDLVFQYVEDPTIVRIEPEWSIVSGNTPIAVWGTHLDLIQNPQIRAKHGGKEHINICEVLNATEMTCQAPALALGPDHQSDLTERPEEFGFILDNVQSLLILNKTNFTYYPNPVFEAFGPSGILELKPGTPIILKGKNLIPPVAGGNVKLNYTVLVGEKPCTVTVSDVQLLCESPNLIGRHKVMARVGGMEYSPGMVYIAPDSPLSLPAIVSIAVAGGLLIIFIVAVLIAYKRKSRESDLTLKRLQMQMDNLESRVALECKEAFAELQTDIHELTSDLDGAGIPFLDYRTYTMRVLFPGIEDHPVLRDLEVPGYRQERVEKGLKLFAQLINNKVFLLSFIRTLESQRSFSMRDRGNVASLIMTVLQSKLEYATDVLKQLLADLIDKNLESKNHPKLLLRRTESVAEKMLTNWFTFLLYKFLKECAGEPLFSLFCAIKQQMEKGPIDAITGEARYSLSEDKLIRQQIDYKTLVLSCVSPDNANSPEVPVKILNCDTITQVKEKILDAIFKNVPCSHRPKAADMDLEWRQGSGARMILQDEDITTKIENDWKRLNTLAHYQVPDGSVVALVSKQVTAYNAVNNSTVSRTSASKYENMIRYTGSPDSLRSRTPMITPDLESGVKMWHLVKNHEHGDQKEGDRGSKMVSEIYLTRLLATKGTLQKFVDDLFETIFSTAHRGSALPLAIKYMFDFLDEQADKHGIHDPHVRHTWKSNCLPLRFWVNMIKNPQFVFDIHKNSITDACLSVVAQTFMDSCSTSEHRLGKDSPSNKLLYAKDIPSYKNWVERYYSDIGKMPAISDQDMNAYLAEQSRMHMNEFNTMSALSEIFSYVGKYSEEILGPLDHDDQCGKQKLAYKLEQVITLMSLDS",
# FGF22_HUMAN
"MRRRLWLGLAWLLLARAPDAAGTPSASRGPRSYPHLEGDVRWRRLFSSTHFFLRVDPGGRVQGTRWRHGQDSILEIRSVHVGVVVIKAVSSGFYVAMNRRGRLYGSRLYTVDCRFRERIEENGHNTYASQRWRRRGQPMFLALDRRGGPRPGGRTRRYHLSAHFLPVLVS",
# NRX3B_HUMAN
"MHLRIHARRSPPRRPAWTLGIWFLFWGCIVSSVWSSSNVASSSSTSSSPGSHSQHEHHFHGSKHHSVPISIYRSPVSLRGGHAGATYIFGKSGGLILYTWPANDRPSTRSDRLAVGFSTTVKDGILVRIDSAPGLGDFLQLHIEQGKIGVVFNIGTVDISIKEERTPVNDGKYHVVRFTRNGGNATLQVDNWPVNEHYPTGRQLTIFNTQAQIAIGGKDKGRLFQGQLSGLYYDGLKVLNMAAENNPNIKINGSVRLVGEVPSILGTTQTTSMPPEMSTTVMETTTTMATTTTRKNRSTASIQPTSDDLVSSAECSSDDEDFVECEPSTGGELVIPLLVEDPLATPPIATRAPSITLPPTFRPLLTIIETTKDSLSMTSEAGLPCLSDQGSDGCDDDGLVISGYGSGETFDSNLPPTDDEDFYTTFSLVTDKSLSTSIFEGGYKAHAPKWESKDFRPNKVSETSRTTTTSLSPELIRFTASSSSGMVPKLPAGKMNNRDLKPQPDIVLLPLPTAYELDSTKLKSPLITSPMFRNVPTANPTEPGIRRVPGASEVIRESSSTTGMVVGIVAAAALCILILLYAMYKYRNRDEGSYQVDETRNYISNSAQSNGTLMKEKQQSSKSGHKKQKNKDREYYV",
# IRPL2_HUMAN
"MKPPFLLALVVCSVVSTNLKMVSKRNSVDGCIDWSVDLKTYMALAGEPVRVKCALFYSYIRTNYSTAQSTGLRLMWYKNKGDLEEPIIFSEVRMSKEEDSIWFHSAEAQDSGFYTCVLRNSTYCMKVSMSLTVAENESGLCYNSRIRYLEKSEVTKRKEISCPDMDDFKKSDQEPDVVWYKECKPKMWRSIIIQKGNALLIQEVQEEDGGNYTCELKYEGKLVRRTTELKVTALLTDKPPKPLFPMENQPSVIDVQLGKPLNIPCKAFFGFSGESGPMIYWMKGEKFIEELAGHIREGEIRLLKEHLGEKEVELALIFDSVVEADLANYTCHVENRNGRKHASVLLRKKDLIYKIELAGGLGAIFLLLVLLVVIYKCYNIELMLFYRQHFGADETNDDNKEYDAYLSYTKVDQDTLDCDNPEEEQFALEVLPDVLEKHYGYKLFIPERDLIPSGTYMEDLTRYVEQSRRLIIVLTPDYILRRGWSIFELESRLHNMLVSGEIKVILIECTELKGKVNCQEVESLKRSIKLLSLIKWKGSKSSKLNSKFWKHLVYEMPIKKKEMLPRCHVLDSAEQGLFGELQPIPSIAMTSTSATLVSSQADLPEFHPSDSMQIRHCCRGYKHEIPATTLPVPSLGNHHTYCNLPLTLLNGQLPLNNTLKDTQEFHRNSSLLPLSSKELSFTSDIW",
# ARFG3_HUMAN
"MGDPSKQDILTIFKRLRSVPTNKVCFDCGAKNPSWASITYGVFLCIDCSGSHRSLGVHLSFIRSTELDSNWSWFQLRCMQVGGNASASSFFHQHGCSTNDTNAKYNSRAAQLYREKIKSLASQATRKHGTDLWLDSCVVPPLSPPPKEEDFFASHVSPEVSDTAWASAIAEPSSLTSRPVETTLENNEGGQEQGPSVEGLNVPTKATLEVSSIIKKKPNQAKKGLGAKKGSLGAQKLANTCFNEIEKQAQAADKMKEQEDLAKVVSKEESIVSSLRLAYKDLEIQMKKDEKMNISGKKNVDSDRLGMGFGNCRSVISHSVTSDMQTIEQESPIMAKPRKKYNDDSDDSYFTSSSSYFDEPVELRSSSFSSWDDSSDSYWKKETSKDTETVLKTTGYSDRPTARRKPDYEPVENTDEAQKKFGNVKAISSDMYFGRQSQADYETRARLERLSASSSISSADLFEEPRKQPAGNYSLSSVLPNAPDMAQFKQGVRSVAGKLSVFANGVVTSIQDRYGS",
# RAB18_HUMAN
"MDEDVLTTLKILIIGESGVGKSSLLLRFTDDTFDPELAATIGVDFKVKTISVDGNKAKLAIWDTAGQERFRTLTPSYYRGAQGVILVYDVTRRDTFVKLDNWLNELETYCTRNDIVNMLVGNKIDKENREVDRNEGLKFARKHSMLFIEASAKTCDGVQCAFEELVEKIIQTPGLWESENQNKGVKLSHREEGQGGGACGGYCSVL",
# NRN1_HUMAN
"MGLKLNGRYISLILAVQIAYLVQAVRAAGKCDAVFKGFSDCLLKLGDSMANYPQGLDDKTNIKTVCTYWEDFHSCTVTALTDCQEGAKDMWDKLRKESKNLNIQGSLFELCGSGNGAAGSLLPAFPVLLVSLSAALATWLSF",
# FZD3_HUMAN
"MAMTWIVFSLWPLTVFMGHIGGHSLFSCEPITLRMCQDLPYNTTFMPNLLNHYDQQTAALAMEPFHPMVNLDCSRDFRPFLCALYAPICMEYGRVTLPCRRLCQRAYSECSKLMEMFGVPWPEDMECSRFPDCDEPYPRLVDLNLAGEPTEGAPVAVQRDYGFWCPRELKIDPDLGYSFLHVRDCSPPCPNMYFRREELSFARYFIGLISIICLSATLFTFLTFLIDVTRFRYPERPIIFYAVCYMMVSLIFFIGFLLEDRVACNASIPAQYKASTVTQGSHNKACTMLFMILYFFTMAGSVWWVILTITWFLAAVPKWGSEAIEKKALLFHASAWGIPGTLTIILLAMNKIEGDNISGVCFVGLYDVDALRYFVLAPLCLYVVVGVSLLLAGIISLNRVRIEIPLEKENQDKLVKFMIRIGVFSILYLVPLLVVIGCYFYEQAYRGIWETTWIQERCREYHIPCPYQVTQMSRPDLILFLMKYLMALIVGIPSVFWVGSKKTCFEWASFFHGRRKKEIVNESRQVLQEPDFAQSLLRDPNTPIIRKSRGTSTQGTSTHASSTQLAMVDDQRSKAGSIHSKVSSYHGSLHRSRDGRYTPCSYRGMEERLPHGSMSRLTDHSRHSSSHRLNEQSRHSSIRDLSNNPMTHITHGTSMNRVIEEDGTSA",
# IL1AP_HUMAN
"MTLLWCVVSLYFYGILQSDASERCDDWGLDTMRQIQVFEDEPARIKCPLFEHFLKFNYSTAHSAGLTLIWYWTRQDRDLEEPINFRLPENRISKEKDVLWFRPTLLNDTGNYTCMLRNTTYCSKVAFPLEVVQKDSCFNSPMKLPVHKLYIEYGIQRITCPNVDGYFPSSVKPTITWYMGCYKIQNFNNVIPEGMNLSFLIALISNNGNYTCVVTYPENGRTFHLTRTLTVKVVGSPKNAVPPVIHSPNDHVVYEKEPGEELLIPCTVYFSFLMDSRNEVWWTIDGKKPDDITIDVTINESISHSRTEDETRTQILSIKKVTSEDLKRSYVCHARSAKGEVAKAAKVKQKVPAPRYTVELACGFGATVLLVVILIVVYHVYWLEMVLFYRAHFGTDETILDGKEYDIYVSYARNAEEEEFVLLTLRGVLENEFGYKLCIFDRDSLPGGIVTDETLSFIQKSRRLLVVLSPNYVLQGTQALLELKAGLENMASRGNINVILVQYKAVKETKVKELKRAKTVLTVIKWKGEKSKYPQGRFWKQLQVAMPVKKSPRRSSSDEQGLSYSSLKNV",
# SEM4B_HUMAN
"MLRTAMGLRSWLAAPWGALPPRPPLLLLLLLLLLLQPPPPTWALSPRISLPLGSEERPFLRFEAEHISNYTALLLSRDGRTLYVGAREALFALSSNLSFLPGGEYQELLWGADAEKKQQCSFKGKDPQRDCQNYIKILLPLSGSHLFTCGTAAFSPMCTYINMENFTLARDEKGNVLLEDGKGRCPFDPNFKSTALVVDGELYTGTVSSFQGNDPAISRSQSLRPTKTESSLNWLQDPAFVASAYIPESLGSLQGDDDKIYFFFSETGQEFEFFENTIVSRIARICKGDEGGERVLQQRWTSFLKAQLLCSRPDDGFPFNVLQDVFTLSPSPQDWRDTLFYGVFTSQWHRGTTEGSAVCVFTMKDVQRVFSGLYKEVNRETQQWYTVTHPVPTPRPGACITNSARERKINSSLQLPDRVLNFLKDHFLMDGQVRSRMLLLQPQARYQRVAVHRVPGLHHTYDVLFLGTGDGRLHKAVSVGPRVHIIEELQIFSSGQPVQNLLLDTHRGLLYAASHSGVVQVPMANCSLYRSCGDCLLARDPYCAWSGSSCKHVSLYQPQLATRPWIQDIEGASAKDLCSASSVVSPSFVPTGEKPCEQVQFQPNTVNTLACPLLSNLATRLWLRNGAPVNASASCHVLPTGDLLLVGTQQLGEFQCWSLEEGFQQLVASYCPEVVEDGVADQTDEGGSVPVIISTSRVSAPAGGKASWGADRSYWKEFLVMCTLFVLAVLLPVLFLLYRHRNSMKVFLKQGECASVHPKTCPVVLPPETRPLNGLGPPSTPLDHRGYQSLSDSPPGSRVFTESEKRPLSIQDSFVEVSPVCPRPRVRLGSEIRDSVV",
# PLCB1_HUMAN
"MAGAQPGVHALQLKPVCVSDSLKKGTKFVKWDDDSTIVTPIILRTDPQGFFFYWTDQNKETELLDLSLVKDARCGRHAKAPKDPKLRELLDVGNIGRLEQRMITVVYGPDLVNISHLNLVAFQEEVAKEWTNEVFSLATNLLAQNMSRDAFLEKAYTKLKLQVTPEGRIPLKNIYRLFSADRKRVETALEACSLPSSRNDSIPQEDFTPEVYRVFLNNLCPRPEIDNIFSEFGAKSKPYLTVDQMMDFINLKQRDPRLNEILYPPLKQEQVQVLIEKYEPNNSLARKGQISVDGFMRYLSGEENGVVSPEKLDLNEDMSQPLSHYFINSSHNTYLTAGQLAGNSSVEMYRQVLLSGCRCVELDCWKGRTAEEEPVITHGFTMTTEISFKEVIEAIAECAFKTSPFPILLSFENHVDSPKQQAKMAEYCRLIFGDALLMEPLEKYPLESGVPLPSPMDLMYKILVKNKKKSHKSSEGSGKKKLSEQASNTYSDSSSMFEPSSPGAGEADTESDDDDDDDDCKKSSMDEGTAGSEAMATEEMSNLVNYIQPVKFESFEISKKRNKSFEMSSFVETKGLEQLTKSPVEFVEYNKMQLSRIYPKGTRVDSSNYMPQLFWNAGCQMVALNFQTMDLAMQINMGMYEYNGKSGYRLKPEFMRRPDKHFDPFTEGIVDGIVANTLSVKIISGQFLSDKKVGTYVEVDMFGLPVDTRRKAFKTKTSQGNAVNPVWEEEPIVFKKVVLPTLACLRIAVYEEGGKFIGHRILPVQAIRPGYHYICLRNERNQPLTLPAVFVYIEVKDYVPDTYADVIEALSNPIRYVNLMEQRAKQLAALTLEDEEEVKKEADPGETPSEAPSEARTTPAENGVNHTTTLTPKPPSQALHSQPAPGSVKAPAKTEDLIQSVLTEVEAQTIEELKQQKSFVKLQKKHYKEMKDLVKRHHKKTTDLIKEHTTKYNEIQNDYLRRRAALEKSAKKDSKKKSEPSSPDHGSSTIEQDLAALDAEMTQKLIDLKDKQQQQLLNLRQEQYYSEKYQKREHIKLLIQKLTDVAEECQNNQLKKLKEICEKEKKELKKKMDKKRQEKITEAKSKDKSQMEEEKTEMIRSYIQEVVQYIKRLEEAQSKRQEKLVEKHKEIRQQILDEKPKLQVELEQEYQDKFKRLPLEILEFVQEAMKGKISEDSNHGSAPLSLSSDPGKVNHKTPSSEELGGDIPGKEFDTPL",
# CRAC1_HUMAN
"MAPSADPGMSRMLPFLLLLWFLPITEGSQRAEPMFTAVTNSVLPPDYDSNPTQLNYGVAVTDVDHDGDFEIVVAGYNGPNLVLKYDRAQKRLVNIAVDERSSPYYALRDRQGNAIGVTACDIDGDGREEIYFLNTNNAFSGVATYTDKLFKFRNNRWEDILSDEVNVARGVASLFAGRSVACVDRKGSGRYSIYIANYAYGNVGPDALIEMDPEASDLSRGILALRDVAAEAGVSKYTGGRGVSVGPILSSSASDIFCDNENGPNFLFHNRGDGTFVDAAASAGVDDPHQHGRGVALADFNRDGKVDIVYGNWNGPHRLYLQMSTHGKVRFRDIASPKFSMPSPVRTVITADFDNDQELEIFFNNIAYRSSSANRLFRVIRREHGDPLIEELNPGDALEPEGRGTGGVVTDFDGDGMLDLILSHGESMAQPLSVFRGNQGFNNNWLRVVPRTRFGAFARGAKVVLYTKKSGAHLRIIDGGSGYLCEMEPVAHFGLGKDEASSVEVTWPDGKMVSRNVASGEMNSVLEILYPRDEDTLQDPAPLECGQGFSQQENGHCMDTNECIQFPFVCPRDKPVCVNTYGSYRCRTNKKCSRGYEPNEDGTACVGTLGQSPGPRPTTPTAAAATAAAAAAAGAATAAPVLVDGDLNLGSVVKESCEPSC",
# ANO2_HUMAN
"MATPGPRDIPLLPGSPRRLSPQAGSRGGQGPKHGQQCLKMPGPRAPGLQGGSNRDPGQPCGGESTRSSSVINNYLDANEPVSLEARLSRMHFHDSQRKVDYVLAYHYRKRGVHLAQGFPGHSLAIVSNGETGKEPHAGGPGDIELGPLDALEEERKEQREEFEHNLMEAGLELEKDLENKSQGSIFVRIHAPWQVLAREAEFLKIKVPTKKEMYEIKAGGSIAKKFSAALQKLSSHLQPRVPEHSNNKMKNLSYPFSREKMYLYNIQEKDTFFDNATRSRIVHEILKRTACSRANNTMGINSLIANNIYEAAYPLHDGEYDSPEDDMNDRKLLYQEWARYGVFYKFQPIDLIRKYFGEKIGLYFAWLGLYTSFLIPSSVIGVIVFLYGCATIEEDIPSREMCDQQNAFTMCPLCDKSCDYWNLSSACGTAQASHLFDNPATVFFSIFMALWATMFLENWKRLQMRLGYFWDLTGIEEEEERAQEHSRPEYETKVREKMLKESNQSAVQKLETNTTECGDEDDEDKLTWKDRFPGYLMNFASILFMIALTFSIVFGVIVYRITTAAALSLNKATRSNVRVTVTATAVIINLVVILILDEIYGAVAKWLTKIEVPKTEQTFEERLILKAFLLKFVNAYSPIFYVAFFKGRFVGRPGSYVYVFDGYRMEECAPGGCLMELCIQLSIIMLGKQLIQNNIFEIGVPKLKKLFRKLKDETEAGETDSAHSKHPEQWDLDYSLEPYTGLTPEYMEMIIQFGFVTLFVASFPLAPVFALLNNVIEVRLDAKKFVTELRRPDAVRTKDIGIWFDILSGIGKFSVISNAFVIAITSDFIPRLVYQYSYSHNGTLHGFVNHTLSFFNVSQLKEGTQPENSQFDQEVQFCRFKDYREPPWAPNPYEFSKQYWFILSARLAFVIIFQNLVMFLSVLVDWMIPDIPTDISDQIKKEKSLLVDFFLKEEHEKLKLMDEPALRSPGGGDRSRSRAASSAPSGQSQLGSMMSSGSQHTNV",
# RTN4_HUMAN
"MEDLDQSPLVSSSDSPPRPQPAFKYQFVREPEDEEEEEEEEEEDEDEDLEELEVLERKPAAGLSAAPVPTAPAAGAPLMDFGNDFVPPAPRGPLPAAPPVAPERQPSWDPSPVSSTVPAPSPLSAAAVSPSKLPEDDEPPARPPPPPPASVSPQAEPVWTPPAPAPAAPPSTPAAPKRRGSSGSVDETLFALPAASEPVIRSSAENMDLKEQPGNTISAGQEDFPSVLLETAASLPSLSPLSAASFKEHEYLGNLSTVLPTEGTLQENVSEASKEVSEKAKTLLIDRDLTEFSELEYSEMGSSFSVSPKAESAVIVANPREEIIVKNKDEEEKLVSNNILHNQQELPTALTKLVKEDEVVSSEKAKDSFNEKRVAVEAPMREEYADFKPFERVWEVKDSKEDSDMLAAGGKIESNLESKVDKKCFADSLEQTNHEKDSESSNDDTSFPSTPEGIKDRSGAYITCAPFNPAATESIATNIFPLLGDPTSENKTDEKKIEEKKAQIVTEKNTSTKTSNPFLVAAQDSETDYVTTDNLTKVTEEVVANMPEGLTPDLVQEACESELNEVTGTKIAYETKMDLVQTSEVMQESLYPAAQLCPSFEESEATPSPVLPDIVMEAPLNSAVPSAGASVIQPSSSPLEASSVNYESIKHEPENPPPYEEAMSVSLKKVSGIKEEIKEPENINAALQETEAPYISIACDLIKETKLSAEPAPDFSDYSEMAKVEQPVPDHSELVEDSSPDSEPVDLFSDDSIPDVPQKQDETVMLVKESLTETSFESMIEYENKEKLSALPPEGGKPYLESFKLSLDNTKDTLLPDEVSTLSKKEKIPLQMEELSTAVYSNDDLFISKEAQIRETETFSDSSPIEIIDEFPTLISSKTDSFSKLAREYTDLEVSHKSEIANAPDGAGSLPCTELPHDLSLKNIQPKVEEKISFSDDFSKNGSATSKVLLLPPDVSALATQAEIESIVKPKVLVKEAEKKLPSDTEKEDRSPSAIFSAELSKTSVVDLLYWRDIKKTGVVFGASLFLLLSLTVFSIVSVTAYIALALLSVTISFRIYKGVIQAIQKSDEGHPFRAYLESEVAISEELVQKYSNSALGHVNCTIKELRRLFLVDDLVDSLKFAVLMWVFTYVGALFNGLTLLILALISLFSVPVIYERHQAQIDHYLGLANKNVKDAMAKIQAKIPGLKRKAE",
# CYLD_HUMAN
"MSSGLWSQEKVTSPYWEERIFYLLLQECSVTDKQTQKLLKVPKGSIGQYIQDRSVGHSRIPSAKGKKNQIGLKILEQPHAVLFVDEKDVVEINEKFTELLLAITNCEERFSLFKNRNRLSKGLQIDVGCPVKVQLRSGEEKFPGVVRFRGPLLAERTVSGIFFGVELLEEGRGQGFTDGVYQGKQLFQCDEDCGVFVALDKLELIEDDDTALESDYAGPGDTMQVELPPLEINSRVSLKVGETIESGTVIFCDVLPGKESLGYFVGVDMDNPIGNWDGRFDGVQLCSFACVESTILLHINDIIPALSESVTQERRPPKLAFMSRGVGDKGSSSHNKPKATGSTSDPGNRNRSELFYTLNGSSVDSQPQSKSKNTWYIDEVAEDPAKSLTEISTDFDRSSPPLQPPPVNSLTTENRFHSLPFSLTKMPNTNGSIGHSPLSLSAQSVMEELNTAPVQESPPLAMPPGNSHGLEVGSLAEVKENPPFYGVIRWIGQPPGLNEVLAGLELEDECAGCTDGTFRGTRYFTCALKKALFVKLKSCRPDSRFASLQPVSNQIERCNSLAFGGYLSEVVEENTPPKMEKEGLEIMIGKKKGIQGHYNSCYLDSTLFCLFAFSSVLDTVLLRPKEKNDVEYYSETQELLRTEIVNPLRIYGYVCATKIMKLRKILEKVEAASGFTSEEKDPEEFLNILFHHILRVEPLLKIRSAGQKVQDCYFYQIFMEKNEKVGVPTIQQLLEWSFINSNLKFAEAPSCLIIQMPRFGKDFKLFKKIFPSLELNITDLLEDTPRQCRICGGLAMYECRECYDDPDISAGKIKQFCKTCNTQVHLHPKRLNHKYNPVSLPKDLPDWDWRHGCIPCQNMELFAVLCIETSHYVAFVKYGKDDSAWLFFDSMADRDGGQNGFNIPQVTPCPEVGEYLKMSLEDLHSLDSRRIQGCARRLLCDAYMCMYQSPTMSLYK",
# NECT3_HUMAN
"MARTLRPSPLCPGGGKAQLSSASLLGAGLLLQPPTPPPLLLLLFPLLLFSRLCGALAGPIIVEPHVTAVWGKNVSLKCLIEVNETITQISWEKIHGKSSQTVAVHHPQYGFSVQGEYQGRVLFKNYSLNDATITLHNIGFSDSGKYICKAVTFPLGNAQSSTTVTVLVEPTVSLIKGPDSLIDGGNETVAAICIAATGKPVAHIDWEGDLGEMESTTTSFPNETATIISQYKLFPTRFARGRRITCVVKHPALEKDIRYSFILDIQYAPEVSVTGYDGNWFVGRKGVNLKCNADANPPPFKSVWSRLDGQWPDGLLASDNTLHFVHPLTFNYSGVYICKVTNSLGQRSDQKVIYISDPPTTTTLQPTIQWHPSTADIEDLATEPKKLPFPLSTLATIKDDTIATIIASVVGGALFIVLVSVLAGIFCYRRRRTFRGDYFAKNYIPPSDMQKESQIDVLQQDELDSYPDSVKKENKNPVNNLIRKDYLEEPEKTQWNNVENLNRFERPMDYYEDLKMGMKFVSDEHYDENEDDLVSHVDGSVISRREWYV",
# PAK6_HUMAN
"MFRKKKKKRPEISAPQNFQHRVHTSFDPKEGKFVGLPPQWQNILDTLRRPKPVVDPSRITRVQLQPMKTVVRGSAMPVDGYISGLLNDIQKLSVISSNTLRGRSPTSRRRAQSLGLLGDEHWATDPDMYLQSPQSERTDPHGLYLSCNGGTPAGHKQMPWPEPQSPRVLPNGLAAKAQSLGPAEFQGASQRCLQLGACLQSSPPGASPPTGTNRHGMKAAKHGSEEARPQSCLVGSATGRPGGEGSPSPKTRESSLKRRLFRSMFLSTAATAPPSSSKPGPPPQSKPNSSFRPPQKDNPPSLVAKAQSLPSDQPVGTFSPLTTSDTSSPQKSLRTAPATGQLPGRSSPAGSPRTWHAQISTSNLYLPQDPTVAKGALAGEDTGVVTHEQFKAALRMVVDQGDPRLLLDSYVKIGEGSTGIVCLAREKHSGRQVAVKMMDLRKQQRRELLFNEVVIMRDYQHFNVVEMYKSYLVGEELWVLMEFLQGGALTDIVSQVRLNEEQIATVCEAVLQALAYLHAQGVIHRDIKSDSILLTLDGRVKLSDFGFCAQISKDVPKRKSLVGTPYWMAPEVISRSLYATEVDIWSLGIMVIEMVDGEPPYFSDSPVQAMKRLRDSPPPKLKNSHKVSPVLRDFLERMLVRDPQERATAQELLDHPFLLQTGLPECLVPLIQLYRKQTSTC",
# GEPH_HUMAN
"MATEGMILTNHDHQIRVGVLTVSDSCFRNLAEDRSGINLKDLVQDPSLLGGTISAYKIVPDEIEEIKETLIDWCDEKELNLILTTGGTGFAPRDVTPEATKEVIEREAPGMALAMLMGSLNVTPLGMLSRPVCGIRGKTLIINLPGSKKGSQECFQFILPALPHAIDLLRDAIVKVKEVHDELEDLPSPPPPLSPPPTTSPHKQTEDKGVQCEEEEEEKKDSGVASTEDSSSSHITAAAIAAKIPDSIISRGVQVLPRDTASLSTTPSESPRAQATSRLSTASCPTPKVQSRCSSKENILRASHSAVDITKVARRHRMSPFPLTSMDKAFITVLEMTPVLGTEIINYRDGMGRVLAQDVYAKDNLPPFPASVKDGYAVRAADGPGDRFIIGESQAGEQPTQTVMPGQVMRVTTGAPIPCGADAVVQVEDTELIRESDDGTEELEVRILVQARPGQDIRPIGHDIKRGECVLAKGTHMGPSEIGLLATVGVTEVEVNKFPVVAVMSTGNELLNPEDDLLPGKIRDSNRSTLLATIQEHGYPTINLGIVGDNPDDLLNALNEGISRADVIITSGGVSMGEKDYLKQVLDIDLHAQIHFGRVFMKPGLPTTFATLDIDGVRKIIFALPGNPVSAVVTCNLFVVPALRKMQGILDPRPTIIKARLSCDVKLDPRPEYHRCILTWHHQEPLPWAQSTGNQMSSRLMSMRSANGLLMLPPKTEQYVELHKGEVVDVMVIGRL",
# ANM8_HUMAN
"MGMKHSSRCLLLRRKMAENAAESTEVNSPPSQPPQPVVPAKPVQCVHHVSTQPSCPGRGKMSKLLNPEEMTSRDYYFDSYAHFGIHEEMLKDEVRTLTYRNSMYHNKHVFKDKVVLDVGSGTGILSMFAAKAGAKKVFGIECSSISDYSEKIIKANHLDNIITIFKGKVEEVELPVEKVDIIISEWMGYCLFYESMLNTVIFARDKWLKPGGLMFPDRAALYVVAIEDRQYKDFKIHWWENVYGFDMTCIRDVAMKEPLVDIVDPKQVVTNACLIKEVDIYTVKTEELSFTSAFCLQIQRNDYVHALVTYFNIEFTKCHKKMGFSTAPDAPYTHWKQTVFYLEDYLTVRRGEEIYGTISMKPNAKNVRDLDFTVDLDFKGQLCETSVSNDYKMR",
# SHLB2_HUMAN
"MDFNMKKLASDAGIFFTRAVQFTEEKFGQAEKTELDAHFENLLARADSTKNWTEKILRQTEVLLQPNPSARVEEFLYEKLDRKVPSRVTNGELLAQYMADAASELGPTTPYGKTLIKVAEAEKQLGAAERDFIHTASISFLTPLRNFLEGDWKTISKERRLLQNRRLDLDACKARLKKAKAAEAKATTVPDFQETRPRNYILSASASALWNDEVDKAEQELRVAQTEFDRQAEVTRLLLEGISSTHVNHLRCLHEFVKSQTTYYAQCYRHMLDLQKQLGRFPGTFVGTTEPASPPLSSTSPTTAAATMPVVPSVASLAPPGEASLCLEEVAPPASGTRKARVLYDYEAADSSELALLADELITVYSLPGMDPDWLIGERGNKKGKVPVTYLELLS",
# KCNQ5_HUMAN
"MPRHHAGGEEGGAAGLWVKSGAAAAAAGGGRLGSGMKDVESGRGRVLLNSAAARGDGLLLLGTRAATLGGGGGGLRESRRGKQGARMSLLGKPLSYTSSQSCRRNVKYRRVQNYLYNVLERPRGWAFIYHAFVFLLVFGCLILSVFSTIPEHTKLASSCLLILEFVMIVVFGLEFIIRIWSAGCCCRYRGWQGRLRFARKPFCVIDTIVLIASIAVVSAKTQGNIFATSALRSLRFLQILRMVRMDRRGGTWKLLGSVVYAHSKELITAWYIGFLVLIFSSFLVYLVEKDANKEFSTYADALWWGTITLTTIGYGDKTPLTWLGRLLSAGFALLGISFFALPAGILGSGFALKVQEQHRQKHFEKRRNPAANLIQCVWRSYAADEKSVSIATWKPHLKALHTCSPTKKEQGEASSSQKLSFKERVRMASPRGQSIKSRQASVGDRRSPSTDITAEGSPTKVQKSWSFNDRTRFRPSLRLKSSQPKPVIDADTALGTDDVYDEKGCQCDVSVEDLTPPLKTVIRAIRIMKFHVAKRKFKETLRPYDVKDVIEQYSAGHLDMLCRIKSLQTRVDQILGKGQITSDKKSREKITAEHETTDDLSMLGRVVKVEKQVQSIESKLDCLLDIYQQVLRKGSASALALASFQIPPFECEQTSDYQSPVDSKDLSGSAQNSGCLSRSTSANISRGLQFILTPNEFSAQTFYALSPTMHSQATQVPISQSDGSAVAATNTIANQINTAPKPAAPTTLQIPPPLPAIKHLPRPETLHPNPAGLQESISDVTTCLVASKENVQVAQSNLTKDRSMRKSFDMGGETLLSVCPMVPKDLGKSLSVQNLIRSTEELNIQLSGSESSGSRGSQDFYPKWRESKLFITDEEVGPEETETDTFDAAPQPAREAAFASDSLRTGRSRSSQSICKAGESTDALSLPHVKLK",
# S17A5_HUMAN
"MRSPVRDLARNDGEESTDRTPLLPGAPRAEAAPVCCSARYNLAILAFFGFFIVYALRVNLSVALVDMVDSNTTLEDNRTSKACPEHSAPIKVHHNQTGKKYQWDAETQGWILGSFFYGYIITQIPGGYVASKIGGKMLLGFGILGTAVLTLFTPIAADLGVGPLIVLRALEGLGEGVTFPAMHAMWSSWAPPLERSKLLSISYAGAQLGTVISLPLSGIICYYMNWTYVFYFFGTIGIFWFLLWIWLVSDTPQKHKRISHYEKEYILSSLRNQLSSQKSVPWVPILKSLPLWAIVVAHFSYNWTFYTLLTLLPTYMKEILRFNVQENGFLSSLPYLGSWLCMILSGQAADNLRAKWNFSTLCVRRIFSLIGMIGPAVFLVAAGFIGCDYSLAVAFLTISTTLGGFCSSGFSINHLDIAPSYAGILLGITNTFATIPGMVGPVIAKSLTPDNTVGEWQTVFYIAAAINVFGAIFFTLFAKGEVQNWALNDHHGHRH",
# PICK1_HUMAN
"MFADLDYDIEEDKLGIPTVPGKVTLQKDAQNLIGISIGGGAQYCPCLYIVQVFDNTPAALDGTVAAGDEITGVNGRSIKGKTKVEVAKMIQEVKGEVTIHYNKLQADPKQGMSLDIVLKKVKHRLVENMSSGTADALGLSRAILCNDGLVKRLEELERTAELYKGMTEHTKNLLRAFYELSQTHRAFGDVFSVIGVREPQPAASEAFVKFADAHRSIEKFGIRLLKTIKPMLTDLNTYLNKAIPDTRLTIKKYLDVKFEYLSYCLKVKEMDDEEYSCIALGEPLYRVSTGNYEYRLILRCRQEARARFSQMRKDVLEKMELLDQKHVQDIVFQLQRLVSTMSKYYNDCYAVLRDADVFPIEVDLAHTTLAYGLNQEEFTDGEEEEEEEDTAAGEPSRDTRGAAGPLDKGGSWCDS",
# DISC1_HUMAN
"MPGGGPQGAPAAAGGGGVSHRAGSRDCLPPAACFRRRRLARRPGYMRSSTGPGIGFLSPAVGTLFRFPGGVSGEESHHSESRARQCGLDSRGLLVRSPVSKSAAAPTVTSVRGTSAHFGIQLRGGTRLPDRLSWPCGPGSAGWQQEFAAMDSSETLDASWEAACSDGARRVRAAGSLPSAELSSNSCSPGCGPEVPPTPPGSHSAFTSSFSFIRLSLGSAGERGEAEGCPPSREAESHCQSPQEMGAKAASLDGPHEDPRCLSRPFSLLATRVSADLAQAARNSSRPERDMHSLPDMDPGSSSSLDPSLAGCGGDGSSGSGDAHSWDTLLRKWEPVLRDCLLRNRRQMEVISLRLKLQKLQEDAVENDDYDKAETLQQRLEDLEQEKISLHFQLPSRQPALSSFLGHLAAQVQAALRRGATQQASGDDTHTPLRMEPRLLEPTAQDSLHVSITRRDWLLQEKQQLQKEIEALQARMFVLEAKDQQLRREIEEQEQQLQWQGCDLTPLVGQLSLGQLQEVSKALQDTLASAGQIPFHAEPPETIRSLQERIKSLNLSLKEITTKVCMSEKFCSTLRKKVNDIETQLPALLEAKMHAISGNHFWTAKDLTEEIRSLTSEREGLEGLLSKLLVLSSRNVKKLGSVKEDYNRLRREVEHQETAYETSVKENTMKYMETLKNKLCSCKCPLLGKVWEADLEACRLLIQSLQLQEARGSLSVEDERQMDDLEGAAPPIPPRLHSEDKRKTPLKVLEEWKTHLIPSLHCAGGEQKEESYILSAELGEKCEDIGKKLLYLEDQLHTAIHSHDEDLIQSLRRELQMVKETLQAMILQLQPAKEAGEREAAASCMTAGVHEAQA",
# STRN4_HUMAN
"MMEERAAAAVAAAASSCRPLGSGAGPGPTGAAPVSAPAPGPGPAGKGGGGGGSPGPTAGPEPLSLPGILHFIQHEWARFEAEKARWEAERAELQAQVAFLQGERKGQENLKTDLVRRIKMLEYALKQERAKYHKLKFGTDLNQGEKKADVSEQVSNGPVESVTLENSPLVWKEGRQLLRQYLEEVGYTDTILDMRSKRVRSLLGRSLELNGAVEPSEGAPRAPPGPAGLSGGESLLVKQIEEQIKRNAAGKDGKERLGGSVLGQIPFLQNCEDEDSDEDDELDSVQHKKQRVKLPSKALVPEMEDEDEEDDSEDAINEFDFLGSGEDGEGAPDPRRCTVDGSPHELESRRVKLQGILADLRDVDGLPPKVTGPPPGTPQPRPHEDVFIMDTIGGGEVSLGDLADLTVTNDNDLSCDLSDSKDAFKKTWNPKFTLRSHYDGIRSLAFHHSQSALLTASEDGTLKLWNLQKAVTAKKNAALDVEPIHAFRAHRGPVLAVAMGSNSEYCYSGGADACIHSWKIPDLSMDPYDGYDPSVLSHVLEGHGDAVWGLAFSPTSQRLASCSADGTVRIWDPSSSSPACLCTFPTASEHGVPTSVAFTSTEPAHIVASFRSGDTVLYDMEVGSALLTLESRGSSGPTQINQVVSHPNQPLTITAHDDRGIRFLDNRTGKPVHSMVAHLDAVTCLAVDPNGAFLMSGSHDCSLRLWSLDNKTCVQEITAHRKKHEEAIHAVACHPSKALIASAGADALAKVFV",
# RNC_HUMAN
"MMQGNTCHRMSFHPGRGCPRGRGGHGARPSAPSFRPQNLRLLHPQQPPVQYQYEPPSAPSTTFSNSPAPNFLPPRPDFVPFPPPMPPSAQGPLPPCPIRPPFPNHQMRHPFPVPPCFPPMPPPMPCPNNPPVPGAPPGQGTFPFMMPPPSMPHPPPPPVMPQQVNYQYPPGYSHHNFPPPSFNSFQNNPSSFLPSANNSSSPHFRHLPPYPLPKAPSERRSPERLKHYDDHRHRDHSHGRGERHRSLDRRERGRSPDRRRQDSRYRSDYDRGRTPSRHRSYERSRERERERHRHRDNRRSPSLERSYKKEYKRSGRSYGLSVVPEPAGCTPELPGEIIKNTDSWAPPLEIVNHRSPSREKKRARWEEEKDRWSDNQSSGKDKNYTSIKEKEPEETMPDKNEEEEEELLKPVWIRCTHSENYYSSDPMDQVGDSTVVGTSRLRDLYDKFEEELGSRQEKAKAARPPWEPPKTKLDEDLESSSESECESDEDSTCSSSSDSEVFDVIAEIKRKKAHPDRLHDELWYNDPGQMNDGPLCKCSAKARRTGIRHSIYPGEEAIKPCRPMTNNAGRLFHYRITVSPPTNFLTDRPTVIEYDDHEYIFEGFSMFAHAPLTNIPLCKVIRFNIDYTIHFIEEMMPENFCVKGLELFSLFLFRDILELYDWNLKGPLFEDSPPCCPRFHFMPRFVRFLPDGGKEVLSMHQILLYLLRCSKALVPEEEIANMLQWEELEWQKYAEECKGMIVTNPGTKPSSVRIDQLDREQFNPDVITFPIIVHFGIRPAQLSYAGDPQYQKLWKSYVKLRHLLANSPKVKQTDKQKLAQREEALQKIRQKNTMRREVTVELSSQGFWKTGIRSDVCQHAMMLPVLTHHIRYHQCLMHLDKLIGYTFQDRCLLQLAMTHPSHHLNFGMNPDHARNSLSNCGIRQPKYGDRKVHHMHMRKKGINTLINIMSRLGQDDPTPSRINHNERLEFLGDAVVEFLTSVHLYYLFPSLEEGGLATYRTAIVQNQHLAMLAKKLELDRFMLYAHGPDLCRESDLRHAMANCFEALIGAVYLEGSLEEAKQLFGRLLFNDPDLREVWLNYPLHPLQLQEPNTDRQLIETSPVLQKLTEFEEAIGVIFTHVRLLARAFTLRTVGFNHLTLGHNQRMEFLGDSIMQLVATEYLFIHFPDHHEGHLTLLRSSLVNNRTQAKVAEELGMQEYAITNDKTKRPVALRTKTLADLLESFIAALYIDKDLEYVHTFMNVCFFPRLKEFILNQDWNDPKSQLQQCCLTLRTEGKEPDIPLYKTLQTVGPSHARTYTVAVYFKGERIGCGKGPSIQQAEMGAAMDALEKYNFPQMAHQKRFIERKYRQELKEMRWEREHQEREPDETEDIKK",
# RAB6B_HUMAN
"MSAGGDFGNPLRKFKLVFLGEQSVGKTSLITRFMYDSFDNTYQATIGIDFLSKTMYLEDRTVRLQLWDTAGQERFRSLIPSYIRDSTVAVVVYDITNLNSFQQTSKWIDDVRTERGSDVIIMLVGNKTDLADKRQITIEEGEQRAKELSVMFIETSAKTGYNVKQLFRRVASALPGMENVQEKSKEGMIDIKLDKPQEPPASEGGCSC",
# VPS45_HUMAN
"MNVVFAVKQYISKMIEDSGPGMKVLLMDKETTGIVSMVYTQSEILQKEVYLFERIDSQNREIMKHLKAICFLRPTKENVDYIIQELRRPKYTIYFIYFSNVISKSDVKSLAEADEQEVVAEVQEFYGDYIAVNPHLFSLNILGCCQGRNWDPAQLSRTTQGLTALLLSLKKCPMIRYQLSSEAAKRLAECVKQVITKEYELFEFRRTEVPPLLLILDRCDDAITPLLNQWTYQAMVHELLGINNNRIDLSRVPGISKDLREVVLSAENDEFYANNMYLNFAEIGSNIKNLMEDFQKKKPKEQQKLESIADMKAFVENYPQFKKMSGTVSKHVTVVGELSRLVSERNLLEVSEVEQELACQNDHSSALQNIKRLLQNPKVTEFDAARLVMLYALHYERHSSNSLPGLMMDLRNKGVSEKYRKLVSAVVEYGGKRVRGSDLFSPKDAVAITKQFLKGLKGVENVYTQHQPFLHETLDHLIKGRLKENLYPYLGPSTLRDRPQDIIVFVIGGATYEEALTVYNLNRTTPGVRIVLGGTTVHNTKSFLEEVLASGLHSRSKESSQVTSRSASRR",
# HOME2_HUMAN
"MGEQPIFTTRAHVFQIDPNTKKNWMPASKQAVTVSYFYDVTRNSYRIISVDGAKVIINSTITPNMTFTKTSQKFGQWADSRANTVFGLGFSSEQQLTKFAEKFQEVKEAAKIAKDKTQEKIETSSNHSQESGRETPSSTQASSVNGTDDEKASHAGPANTHLKSENDKLKIALTQSAANVKKWEIELQTLRESNARLTTALQESAASVEQWKRQFSICRDENDRLRNKIDELEEQCSEINREKEKNTQLKRRIEELEAELREKETELKDLRKQSEIIPQLMSECEYVSEKLEAAERDNQNLEDKVRSLKTDIEESKYRQRHLKVELKSFLEVLDGKIDDLHDFRRGLSKLGTDN",
# HOME3_HUMAN
"MSTAREQPIFSTRAHVFQIDPATKRNWIPAGKHALTVSYFYDATRNVYRIISIGGAKAIINSTVTPNMTFTKTSQKFGQWADSRANTVYGLGFASEQHLTQFAEKFQEVKEAARLAREKSQDGGELTSPALGLASHQVPPSPLVSANGPGEEKLFRSQSADAPGPTERERLKKMLSEGSVGEVQWEAEFFALQDSNNKLAGALREANAAAAQWRQQLEAQRAEAERLRQRVAELEAQAASEVTPTGEKEGLGQGQSLEQLEALVQTKDQEIQTLKSQTGGPREALEAAEREETQQKVQDLETRNAELEHQLRAMERSLEEARAERERARAEVGRAAQLLDVSLFELSELREGLARLAEAAP",
# TEN2_HUMAN
"MDVKDRRHRSLTRGRCGKECRYTSSSLDSEDCRVPTQKSYSSSETLKAYDHDSRMHYGNRVTDLIHRESDEFPRQGTNFTLAELGICEPSPHRSGYCSDMGILHQGYSLSTGSDADSDTEGGMSPEHAIRLWGRGIKSRRSSGLSSRENSALTLTDSDNENKSDDENGRPIPPTSSPSLLPSAQLPSSHNPPPVSCQMPLLDSNTSHQIMDTNPDEEFSPNSYLLRACSGPQQASSSGPPNHHSQSTLRPPLPPPHNHTLSHHHSSANSLNRNSLTNRRSQIHAPAPAPNDLATTPESVQLQDSWVLNSNVPLETRHFLFKTSSGSTPLFSSSSPGYPLTSGTVYTPPPRLLPRNTFSRKAFKLKKPSKYCSWKCAALSAIAAALLLAILLAYFIAMHLLGLNWQLQPADGHTFNNGIRTGLPGNDDVATMPSGGKVPWSLKNSSIDSGEAEVGRRVTQEVPPGVFWRSQIHISQPQFLKFNISLGKDALFGVYIRRGLPPSHAQYDFMERLDGKEKWSVVESPRERRSIQTLVQNEAVFVQYLDVGLWHLAFYNDGKDKEMVSFNTVVLDSVQDCPRNCHGNGECVSGVCHCFPGFLGADCAKAACPVLCSGNGQYSKGTCQCYSGWKGAECDVPMNQCIDPSCGGHGSCIDGNCVCSAGYKGEHCEEVDCLDPTCSSHGVCVNGECLCSPGWGGLNCELARVQCPDQCSGHGTYLPDTGLCSCDPNWMGPDCSVEVCSVDCGTHGVCIGGACRCEEGWTGAACDQRVCHPRCIEHGTCKDGKCECREGWNGEHCTIGRQTAGTETDGCPDLCNGNGRCTLGQNSWQCVCQTGWRGPGCNVAMETSCADNKDNEGDGLVDCLDPDCCLQSACQNSLLCRGSRDPLDIIQQGQTDWPAVKSFYDRIKLLAGKDSTHIIPGENPFNSSLVSLIRGQVVTTDGTPLVGVNVSFVKYPKYGYTITRQDGTFDLIANGGASLTLHFERAPFMSQERTVWLPWNSFYAMDTLVMKTEENSIPSCDLSGFVRPDPIIISSPLSTFFSAAPGQNPIVPETQVLHEEIELPGSNVKLRYLSSRTAGYKSLLKITMTQSTVPLNLIRVHLMVAVEGHLFQKSFQASPNLAYTFIWDKTDAYGQRVYGLSDAVVSVGFEYETCPSLILWEKRTALLQGFELDPSNLGGWSLDKHHILNVKSGILHKGTGENQFLTQQPAIITSIMGNGRRRSISCPSCNGLAEGNKLLAPVALAVGIDGSLYVGDFNYIRRIFPSRNVTSILELRNKEFKHSNNPAHKYYLAVDPVSGSLYVSDTNSRRIYRVKSLSGTKDLAGNSEVVAGTGEQCLPFDEARCGDGGKAIDATLMSPRGIAVDKNGLMYFVDATMIRKVDQNGIISTLLGSNDLTAVRPLSCDSSMDVAQVRLEWPTDLAVNPMDNSLYVLENNVILRITENHQVSIIAGRPMHCQVPGIDYSLSKLAIHSALESASAIAISHTGVLYITETDEKKINRLRQVTTNGEICLLAGAASDCDCKNDVNCNCYSGDDAYATDAILNSPSSLAVAPDGTIYIADLGNIRIRAVSKNKPVLNAFNQYEAASPGEQELYVFNADGIHQYTVSLVTGEYLYNFTYSTDNDVTELIDNNGNSLKIRRDSSGMPRHLLMPDNQIITLTVGTNGGLKVVSTQNLELGLMTYDGNTGLLATKSDETGWTTFYDYDHEGRLTNVTRPTGVVTSLHREMEKSITIDIENSNRDDDVTVITNLSSVEASYTVVQDQVRNSYQLCNNGTLRVMYANGMGISFHSEPHVLAGTITPTIGRCNISLPMENGLNSIEWRLRKEQIKGKVTIFGRKLRVHGRNLLSIDYDRNIRTEKIYDDHRKFTLRIIYDQVGRPFLWLPSSGLAAVNVSYFFNGRLAGLQRGAMSERTDIDKQGRIVSRMFADGKVWSYSYLDKSMVLLLQSQRQYIFEYDSSDRLLAVTMPSVARHSMSTHTSIGYIRNIYNPPESNASVIFDYSDDGRILKTSFLGTGRQVFYKYGKLSKLSEIVYDSTAVTFGYDETTGVLKMVNLQSGGFSCTIRYRKIGPLVDKQIYRFSEEGMVNARFDYTYHDNSFRIASIKPVISETPLPVDLYRYDEISGKVEHFGKFGVIYYDINQIITTAVMTLSKHFDTHGRIKEVQYEMFRSLMYWMTVQYDSMGRVIKRELKLGPYANTTKYTYDYDGDGQLQSVAVNDRPTWRYSYDLNGNLHLLNPGNSVRLMPLRYDLRDRITRLGDVQYKIDDDGYLCQRGSDIFEYNSKGLLTRAYNKASGWSVQYRYDGVGRRASYKTNLGHHLQYFYSDLHNPTRITHVYNHSNSEITSLYYDLQGHLFAMESSSGEEYYVASDNTGTPLAVFSINGLMIKQLQYTAYGEIYYDSNPDFQMVIGFHGGLYDPLTKLVHFTQRDYDVLAGRWTSPDYTMWKNVGKEPAPFNLYMFKSNNPLSSELDLKNYVTDVKSWLVMFGFQLSNIIPGFPRAKMYFVPPPYELSESQASENGQLITGVQQTTERHNQAFMALEGQVITKKLHASIREKAGHWFATTTPIIGKGIMFAIKEGRVTTGVSSIASEDSRKVASVLNNAYYLDKMHYSIEGKDTHYFVKIGSADGDLVTLGTTIGRKVLESGVNVTVSQPTLLVNGRTRRFTNIEFQYSTLLLSIRYGLTPDTLDEEKARVLDQARQRALGTAWAKEQQKARDGREGSRLWTEGEKQQLLSTGRVQGYEGYYVLPVEQYPELADSSSNIQFLRQNEMGKR",
# LRC4B_HUMAN
"MARARGSPCPPLPPGRMSWPHGALLFLWLFSPPLGAGGGGVAVTSAAGGGSPPATSCPVACSCSNQASRVICTRRDLAEVPASIPVNTRYLNLQENGIQVIRTDTFKHLRHLEILQLSKNLVRKIEVGAFNGLPSLNTLELFDNRLTTVPTQAFEYLSKLRELWLRNNPIESIPSYAFNRVPSLRRLDLGELKRLEYISEAAFEGLVNLRYLNLGMCNLKDIPNLTALVRLEELELSGNRLDLIRPGSFQGLTSLRKLWLMHAQVATIERNAFDDLKSLEELNLSHNNLMSLPHDLFTPLHRLERVHLNHNPWHCNCDVLWLSWWLKETVPSNTTCCARCHAPAGLKGRYIGELDQSHFTCYAPVIVEPPTDLNVTEGMAAELKCRTGTSMTSVNWLTPNGTLMTHGSYRVRISVLHDGTLNFTNVTVQDTGQYTCMVTNSAGNTTASATLNVSAVDPVAAGGTGSGGGGPGGSGGVGGGSGGYTYFTTVTVETLETQPGEEALQPRGTEKEPPGPTTDGVWGGGRPGDAAGPASSSTTAPAPRSSRPTEKAFTVPITDVTENALKDLDDVMKTTKIIIGCFVAITFMAAVMLVAFYKLRKQHQLHKHHGPTRTVEIINVEDELPAASAVSVAAAAAVASGGGVGGDSHLALPALERDHLNHHHYVAAAFKAHYSSNPSGGGCGGKGPPGLNSIHEPLLFKSGSKENVQETQI",
# SAC1_HUMAN
"MATAAYEQLKLHITPEKFYVEACDDGADDVLTIDRVSTEVTLAVKKDVPPSAVTRPIFGILGTIHLVAGNYLIVITKKIKVGEFFSHVVWKATDFDVLSYKKTMLHLTDIQLQDNKTFLAMLNHVLNVDGFYFSTTYDLTHTLQRLSNTSPEFQEMSLLERADQRFVWNGHLLRELSAQPEVHRFALPVLHGFITMHSCSINGKYFDWILISRRSCFRAGVRYYVRGIDSEGHAANFVETEQIVHYNGSKASFVQTRGSIPVFWSQRPNLKYKPLPQISKVANHMDGFQRHFDSQVIIYGKQVIINLINQKGSEKPLEQTFATMVSSLGSGMMRYIAFDFHKECKNMRWDRLSILLDQVAEMQDELSYFLVDSAGQVVANQEGVFRSNCMDCLDRTNVIQSLLARRSLQAQLQRLGVLHVGQKLEEQDEFEKIYKNAWADNANACAKQYAGTGALKTDFTRTGKRTHLGLIMDGWNSMIRYYKNNFSDGFRQDSIDLFLGNYSVDELESHSPLSVPRDWKFLALPIIMVVAFSMCIICLLMAGDTWTETLAYVLFWGVASIGTFFIILYNGKDFVDAPRLVQKEKID",
# CBLN4_HUMAN
"MGSGRRALSAVPAVLLVLTLPGLPVWAQNDTEPIVLEGKCLVVCDSNPATDSKGSSSSPLGISVRAANSKVAFSAVRSTNHEPSEMSNKTRIIYFDQILVNVGNFFTLESVFVAPRKGIYSFSFHVIKVYQSQTIQVNLMLNGKPVISAFAGDKDVTREAATNGVLLYLDKEDKVYLKLEKGNLVGGWQYSTFSGFLVFPL",
# STAU2_HUMAN
"MANPKEKTAMCLVNELARFNRVQPQYKLLNERGPAHSKMFSVQLSLGEQTWESEGSSIKKAQQAVANKALTESTLPKPVQKPPKSNVNNNPGSITPTVELNGLAMKRGEPAIYRPLDPKPFPNYRANYNFRGMYNQRYHCPVPKIFYVQLTVGNNEFFGEGKTRQAARHNAAMKALQALQNEPIPERSPQNGESGKDVDDDKDANKSEISLVFEIALKRNMPVSFEVIKESGPPHMKSFVTRVSVGEFSAEGEGNSKKLSKKRAATTVLQELKKLPPLPVVEKPKLFFKKRPKTIVKAGPEYGQGMNPISRLAQIQQAKKEKEPDYVLLSERGMPRRREFVMQVKVGNEVATGTGPNKKIAKKNAAEAMLLQLGYKASTNLQDQLEKTGENKGWSGPKPGFPEPTNNTPKGILHLSPDVYQEMEASRHKVISGTTLGYLSPKDMNQPSSSFFSISPTSNSSATIARELLMNGTSSTAEAIGLKGSSPTPPCSPVQPSKQLEYLARIQGFQAALSALKQFSEQGLDPIDGAMNIEKGSLEKQAKHLREKADNNQAPPGSIAQDCKKSNSAV",
# LIN7C_HUMAN
"MAALGEPVRLERDICRAIELLEKLQRSGEVPPQKLQALQRVLQSEFCNAVREVYEHVYETVDISSSPEVRANATAKATVAAFAASEGHSHPRVVELPKTEEGLGFNIMGGKEQNSPIYISRIIPGGIADRHGGLKRGDQLLSVNGVSVEGEHHEKAVELLKAAQGKVKLVVRYTPKVLEEMESRFEKMRSAKRRQQT",
# RN19A_HUMAN
"MQEQEIGFISKYNEGLCVNTDPVSILTSILDMSLHRQMGSDRDLQSSASSVSLPSVKKAPKKRRISIGSLFRRKKDNKRKSRELNGGVDGIASIESIHSEMCTDKNSIFSTNTSSDNGLTSISKQIGDFIECPLCLLRHSKDRFPDIMTCHHRSCVDCLRQYLRIEISESRVNISCPECTERFNPHDIRLILSDDVLMEKYEEFMLRRWLVADPDCRWCPAPDCGYAVIAFGCASCPKLTCGREGCGTEFCYHCKQIWHPNQTCDAARQERAQSLRLRTIRSSSISYSQESGAAADDIKPCPRCAAYIIKMNDGSCNHMTCAVCGCEFCWLCMKEISDLHYLSPSGCTFWGKKPWSRKKKILWQLGTLVGAPVGIALIAGIAIPAMIIGIPVYVGRKIHNRYEGKDVSKHKRNLAIAGGVTLSVIVSPVVAAVTVGIGVPIMLAYVYGVVPISLCRSGGCGVSAGNGKGVRIEFDDENDINVGGTNTAVDTTSVAEARHNPSIGEGSVGGLTGSLSASGSHMDRIGAIRDNLSETASTMALAGASITGSLSGSAMVNCFNRLEVQADVQKERYSLSGESGTVSLGTVSDNASTKAMAGSILNSYIPLDKEGNSMEVQVDIESKPSKFRHNSGSSSVDDGSATRSHAGGSSSGLPEGKSSATKWSKEATAGKKSKSGKLRKKGNMKINETREDMDAQLLEQQSTNSSEFEAPSLSDSMPSVADSHSSHFSEFSCSDLESMKTSCSHGSSDYHTRFATVNILPEVENDRLENSPHQCSISVVTQTASCSEVSQLNHIAEEHGNNGIKPNVDLYFGDALKETNNNHSHQTMELKVAIQTEI",
# SEP11_HUMAN
"MAVAVGRPSNEELRNLSLSGHVGFDSLPDQLVNKSTSQGFCFNILCVGETGIGKSTLMDTLFNTKFESDPATHNEPGVRLKARSYELQESNVRLKLTIVDTVGFGDQINKDDSYKPIVEYIDAQFEAYLQEELKIKRSLFNYHDTRIHACLYFIAPTGHSLKSLDLVTMKKLDSKVNIIPIIAKADTIAKNELHKFKSKIMSELVSNGVQIYQFPTDEETVAEINATMSVHLPFAVVGSTEEVKIGNKMAKARQYPWGVVQVENENHCDFVKLREMLIRVNMEDLREQTHTRHYELYRRCKLEEMGFKDTDPDSKPFSLQETYEAKRNEFLGELQKKEEEMRQMFVMRVKEKEAELKEAEKELHEKFDLLKRTHQEEKKKVEDKKKELEEEVNNFQKKKAAAQLLQSQAQQSGAQQTKKDKDKKNASFT",
# ARL8B_HUMAN
"MLALISRLLDWFRSLFWKEEMELTLVGLQYSGKTTFVNVIASGQFSEDMIPTVGFNMRKVTKGNVTIKIWDIGGQPRFRSMWERYCRGVNAIVYMIDAADREKIEASRNELHNLLDKPQLQGIPVLVLGNKRDLPNALDEKQLIEKMNLSAIQDREICCYSISCKEKDNIDITLQWLIQHSKSRRS",
# RN216_HUMAN
"MEEGNNNEEVIHLNNFHCHRGQEWINLRDGPITISDSSDEERIPMLVTPAPQQHEEEDLDDDVILTEDDSEDDYGEFLDLGPPGISEFTKPSGQTEREPKPGPSHNQAANDIVNPRSEQKVIILEEGSLLYTESDPLETQNQSSEDSETELLSNLGESAALADDQAIEEDCWLDHPYFQSLNQQPREITNQVVPQERQPEAELGRLLFQHEFPGPAFPRPEPQQGGISGPSSPQPAHPLGEFEDQQLASDDEEPGPAFPMQESQEPNLENIWGQEAAEVDQELVELLVKETEARFPDVANGFIEEIIHFKNYYDLNVLCNFLLENPDYPKREDRIIINPSSSLLASQDETKLPKIDFFDYSKLTPLDQRCFIQAADLLMADFKVLSSQDIKWALHELKGHYAITRKALSDAIKKWQELSPETSGKRKKRKQMNQYSYIDFKFEQGDIKIEKRMFFLENKRRHCRSYDRRALLPAVQQEQEFYEQKIKEMAEHEDFLLALQMNEEQYQKDGQLIECRCCYGEFPFEELTQCADAHLFCKECLIRYAQEAVFGSGKLELSCMEGSCTCSFPTSELEKVLPQTILYKYYERKAEEEVAAAYADELVRCPSCSFPALLDSDVKRFSCPNPHCRKETCRKCQGLWKEHNGLTCEELAEKDDIKYRTSIEEKMTAARIRKCHKCGTGLIKSEGCNRMSCRCGAQMCYLCRVSINGYDHFCQHPRSPGAPCQECSRCSLWTDPTEDDEKLIEEIQKEAEEEQKRKNGENTFKRIGPPLEKPVEKVQRVEALPRPVPQNLPQPQMPPYAFAHPPFPLPPVRPVFNNFPLNMGPIPAPYVPPLPNVRVNYDFGPIHMPLEHNLPMHFGPQPRHRF",
# SMOX_HUMAN
"MQSCESSGDSADDPLSRGLRRRGQPRVVVIGAGLAGLAAAKALLEQGFTDVTVLEASSHIGGRVQSVKLGHATFELGATWIHGSHGNPIYHLAEANGLLEETTDGERSVGRISLYSKNGVACYLTNHGRRIPKDVVEEFSDLYNEVYNLTQEFFRHDKPVNAESQNSVGVFTREEVRNRIRNDPDDPEATKRLKLAMIQQYLKVESCESSSHSMDEVSLSAFGEWTEIPGAHHIIPSGFMRVVELLAEGIPAHVIQLGKPVRCIHWDQASARPRGPEIEPRGEGDHNHDTGEGGQGGEEPRGGRWDEDEQWSVVVECEDCELIPADHVIVTVSLGVLKRQYTSFFRPGLPTEKVAAIHRLGIGTTDKIFLEFEEPFWGPECNSLQFVWEDEAESHTLTYPPELWYRKICGFDVLYPPERYGHVLSGWICGEEALVMEKCDDEAVAEICTEMLRQFTGNPNIPKPRRILRSAWGSNPYFRGSYSYTQVGSSGADVEKLAKPLPYTESSKTAPMQVLFSGEATHRKYYSTTHGALLSGQREAARLIEMYRDLFQQGT",
# CA2D2_HUMAN
"MAVPARTCGASRPGPARTARPWPGCGPHPGPGTRRPTSGPPRPLWLLLPLLPLLAAPGASAYSFPQQHTMQHWARRLEQEVDGVMRIFGGVQQLREIYKDNRNLFEVQENEPQKLVEKVAGDIESLLDRKVQALKRLADAAENFQKAHRWQDNIKEEDIVYYDAKADAELDDPESEDVERGSKASTLRLDFIEDPNFKNKVNYSYAAVQIPTDIYKGSTVILNELNWTEALENVFMENRRQDPTLLWQVFGSATGVTRYYPATPWRAPKKIDLYDVRRRPWYIQGASSPKDMVIIVDVSGSVSGLTLKLMKTSVCEMLDTLSDDDYVNVASFNEKAQPVSCFTHLVQANVRNKKVFKEAVQGMVAKGTTGYKAGFEYAFDQLQNSNITRANCNKMIMMFTDGGEDRVQDVFEKYNWPNRTVRVFTFSVGQHNYDVTPLQWMACANKGYYFEIPSIGAIRINTQEYLDVLGRPMVLAGKEAKQVQWTNVYEDALGLGLVVTGTLPVFNLTQDGPGEKKNQLILGVMGIDVALNDIKRLTPNYTLGANGYVFAIDLNGYVLLHPNLKPQTTNFREPVTLDFLDAELEDENKEEIRRSMIDGNKGHKQIRTLVKSLDERYIDEVTRNYTWVPIRSTNYSLGLVLPPYSTFYLQANLSDQILQVKLPISKLKDFEFLLPSSFESEGHVFIAPREYCKDLNASDNNTEFLKNFIELMEKVTPDSKQCNNFLLHNLILDTGITQQLVERVWRDQDLNTYSLLAVFAATDGGITRVFPNKAAEDWTENPEPFNASFYRRSLDNHGYVFKPPHQDALLRPLELENDTVGILVSTAVELSLGRRTLRPAVVGVKLDLEAWAEKFKVLASNRTHQDQPQKCGPNSHCEMDCEVNNEDLLCVLIDDGGFLVLSNQNHQWDQVGRFFSEVDANLMLALYNNSFYTRKESYDYQAACAPQPPGNLGAAPRGVFVPTVADFLNLAWWTSAAAWSLFQQLLYGLIYHSWFQADPAEAEGSPETRESSCVMKQTQYYFGSVNASYNAIIDCGNCSRLFHAQRLTNTNLLFVVAEKPLCSQCEAGRLLQKETHSDGPEQCELVQRPRYRRGPHICFDYNATEDTSDCGRGASFPPSLGVLVSLQLLLLLGLPPRPQPQVLVHASRRL",
# SPHK1_HUMAN
"MDPAGGPRGVLPRPCRVLVLLNPRGGKGKALQLFRSHVQPLLAEAEISFTLMLTERRNHARELVRSEELGRWDALVVMSGDGLMHEVVNGLMERPDWETAIQKPLCSLPAGSGNALAASLNHYAGYEQVTNEDLLTNCTLLLCRRLLSPMNLLSLHTASGLRLFSVLSLAWGFIADVDLESEKYRRLGEMRFTLGTFLRLAALRTYRGRLAYLPVGRVGSKTPASPVVVQQGPVDAHLVPLEEPVPSHWTVVPDEDFVLVLALLHSHLGSEMFAAPMGRCAAGVMHLFYVRAGVSRAMLLRLFLAMEKGRHMEYECPYLVYVPVVAFRLEPKDGKGVFAVDGELMVSEAVQGQVHPNYFWMVSGCVEPPPSWKPQQMPPPEEPL",
# ABI2_HUMAN
"MAELQMLLEEEIPGGRRALFDSYTNLERVADYCENNYIQSADKQRALEETKAYTTQSLASVAYLINTLANNVLQMLDIQASQLRRMESSINHISQTVDIHKEKVARREIGILTTNKNTSRTHKIIAPANLERPVRYIRKPIDYTILDDIGHGVKWLLRFKVSTQNMKMGGLPRTTPPTQKPPSPPMSGKGTLGRHSPYRTLEPVRPPVVPNDYVPSPTRNMAPSQQSPVRTASVNQRNRTYSSSGSSGGSHPSSRSSSRENSGSGSVGVPIAVPTPSPPSVFPAPAGSAGTPPLPATSASAPAPLVPATVPSSTAPNAAAGGAPNLADGFTSPTPPVVSSTPPTGHPVQFYSMNRPASRHTPPTIGGSLPYRRPPSITSQTSLQNQMNGGPFYSQNPVSDTPPPPPPVEEPVFDESPPPPPPPEDYEEEEAAVVEYSDPYAEEDPPWAPRSYLEKVVAIYDYTKDKEDELSFQEGAIIYVIKKNDDGWYEGVMNGVTGLFPGNYVESIMHYSE",
# DACT1_HUMAN
"MKPSPAGTAKELEPPAPARGEQRTAEPEGRWREKGEADTERQRTRERQEATLAGLAELEYLRQRQELLVRGALRGAGGAGAAAPRAGELLGEAAQRSRLEEKFLEENILLLRKQLNCLRRRDAGLLNQLQELDKQISDLRLDVEKTSEEHLETDSRPSSGFYELSDGASGSLSNSSNSVFSECLSSCHSSTCFCSPLEATLSLSDGCPKSADLIGLLEYKEGHCEDQASGAVCRSLSTPQFNSLDVIADVNPKYQCDLVSKNGNDVYRYPSPLHAVAVQSPMFLLCLTGNPLREEDRLGNHASDICGGSELDAVKTDSSLPSPSSLWSASHPSSSKKMDGYILSLVQKKTHPVRTNKPRTSVNADPTKGLLRNGSVCVRAPGGVSQGNSVNLKNSKQACLPSGGIPSLNNGTFSPPKQWSKESKAEQAESKRVPLPEGCPSGAASDLQSKHLPKTAKPASQEHARCSAIGTGESPKESAQLSGASPKESPSRGPAPPQENKVVQPLKKMSQKNSLQGVPPATPPLLSTAFPVEERPALDFKSEGSSQSLEEAHLVKAQFIPGQQPSVRLHRGHRNMGVVKNSSLKHRGPALQGLENGLPTVREKTRAGSKKCRFPDDLDTNKKLKKASSKGRKSGGGPEAGVPGRPAGGGHRAGSRAHGHGREAVVAKPKHKRTDYRRWKSSAEISYEEALRRARRGRRENVGLYPAPVPLPYASPYAYVASDSEYSAECESLFHSTVVDTSEDEQSNYTTNCFGDSESSVSEGEFVGESTTTSDSEESGGLIWSQFVQTLPIQTVTAPDLHNHPAKTFVKIKASHNLKKKILRFRSGSLKLMTTV",
# PSD3_HUMAN
"MEGRSAAAETFVWVNNASAHSQSVAKAKYEFLFGRSEGKAPDTSDHGGSTLLPPNVTNEFPEYGTMEEGGEGLRASLEFDGEALPCHPQEQQGVQPLTGCHSGLDSVTEGPKDVREAPSQSHLKEQSLQPIDSLISALKATEARIISGTLQATKVLDQDAVSSFSVQQVEKELDTASRKTQRVNKTLPAGQKNLPEIPLSAEVTTEESFYLSIQKDLTALLTGDTQAEISQIMNNGRKGAVCVQEPSCPLASLGSSAVTCHSAGSVGFLKEQRSALGREHPGGCDRSSSMGRPGRVKHVEFQGVEILWTGGDKRETQHPIDFETSLQRTASPDSKESSKVPRHLISSAGLCNSSSLTENVWDESWKAPSERPGTSSGTFSPVRLDESGEDEVFLQENKQHLEKTPKPERDRERISEQEEHVKGEDEDILGPGYTEDSTDVYSSQFETILDNTSLYYSAESLETLYSEPDSYFSFEMPLTPMIQQRIKEGGQFLERTSGGGHQDILSVSADGGIVMGYSSGVTNGLNDASDSIYTKGTPEIAFWGSNAGVKTTRLEAHSEMGSTEILEKETPENLSNGTSSNVEAAKRLAKRLYQLDRFKRSDVAKHLGKNNEFSKLVAEEYLKFFDFTGMTLDQSLRYFFKAFSLVGETQERERVLIHFSNRYFYCNPDTIASQDGVHCLTCAIMLLNTDLHGHVNIGKKMTCQEFIANLQGVNEGVDFSKDLLKALYNSIKNEKLEWAVDDEEKKKSPSESTEEKANGTHPKTISRIGSTTNPFLDIPHDPNAAVYKSGFLARKIHADMDGKKTPRGKRGWKTFYAVLKGTVLYLQKDEYKPEKALSEEDLKNAVSVHHALASKATDYEKKPNVFKLKTADWRVLLFQTQSPEEMQGWINKINCVAAVFSAPPFPAAIGSQKKFSRPLLPATTTKLSQEEQLKSHESKLKQITTELAEHRSYPPDKKVKAKDVDEYKLKDHYLEFEKTRYEMYVSILKEGGKELLSNDESEAAGLKKSHSSPSLNPDTSPITAKVKRNVSERKDHRPETPSIKQKVT",
# CELR3_HUMAN
"MMARRPPWRGLGGRSTPILLLLLLSLFPLSQEELGGGGHQGWDPGLAATTGPRAHIGGGALALCPESSGVREDGGPGLGVREPIFVGLRGRRQSARNSRGPPEQPNEELGIEHGVQPLGSRERETGQGPGSVLYWRPEVSSCGRTGPLQRGSLSPGALSSGVPGSGNSSPLPSDFLIRHHGPKPVSSQRNAGTGSRKRVGTARCCGELWATGSKGQGERATTSGAERTAPRRNCLPGASGSGPELDSAPRTARTAPASGSAPRESRTAPEPAPKRMRSRGLFRCRFLPQRPGPRPPGLPARPEARKVTSANRARFRRAANRHPQFPQYNYQTLVPENEAAGTAVLRVVAQDPDAGEAGRLVYSLAALMNSRSLELFSIDPQSGLIRTAAALDRESMERHYLRVTAQDHGSPRLSATTMVAVTVADRNDHSPVFEQAQYRETLRENVEEGYPILQLRATDGDAPPNANLRYRFVGPPAARAAAAAAFEIDPRSGLISTSGRVDREHMESYELVVEASDQGQEPGPRSATVRVHITVLDENDNAPQFSEKRYVAQVREDVRPHTVVLRVTATDRDKDANGLVHYNIISGNSRGHFAIDSLTGEIQVVAPLDFEAEREYALRIRAQDAGRPPLSNNTGLASIQVVDINDHIPIFVSTPFQVSVLENAPLGHSVIHIQAVDADHGENARLEYSLTGVAPDTPFVINSATGWVSVSGPLDRESVEHYFFGVEARDHGSPPLSASASVTVTVLDVNDNRPEFTMKEYHLRLNEDAAVGTSVVSVTAVDRDANSAISYQITGGNTRNRFAISTQGGVGLVTLALPLDYKQERYFKLVLTASDRALHDHCYVHINITDANTHRPVFQSAHYSVSVNEDRPMGSTIVVISASDDDVGENARITYLLEDNLPQFRIDADSGAITLQAPLDYEDQVTYTLAITARDNGIPQKADTTYVEVMVNDVNDNAPQFVASHYTGLVSEDAPPFTSVLQISATDRDAHANGRVQYTFQNGEDGDGDFTIEPTSGIVRTVRRLDREAVSVYELTAYAVDRGVPPLRTPVSIQVMVQDVNDNAPVFPAEEFEVRVKENSIVGSVVAQITAVDPDEGPNAHIMYQIVEGNIPELFQMDIFSGELTALIDLDYEARQEYVIVVQATSAPLVSRATVHVRLVDQNDNSPVLNNFQILFNNYVSNRSDTFPSGIIGRIPAYDPDVSDHLFYSFERGNELQLLVVNQTSGELRLSRKLDNNRPLVASMLVTVTDGLHSVTAQCVLRVVIITEELLANSLTVRLENMWQERFLSPLLGRFLEGVAAVLATPAEDVFIFNIQNDTDVGGTVLNVSFSALAPRGAGAGAAGPWFSSEELQEQLYVRRAALAARSLLDVLPFDDNVCLREPCENYMKCVSVLRFDSSAPFLASASTLFRPIQPIAGLRCRCPPGFTGDFCETELDLCYSNPCRNGGACARREGGYTCVCRPRFTGEDCELDTEAGRCVPGVCRNGGTCTDAPNGGFRCQCPAGGAFEGPRCEVAARSFPPSSFVMFRGLRQRFHLTLSLSFATVQQSGLLFYNGRLNEKHDFLALELVAGQVRLTYSTGESNTVVSPTVPGGLSDGQWHTVHLRYYNKPRTDALGGAQGPSKDKVAVLSVDDCDVAVALQFGAEIGNYSCAAAGVQTSSKKSLDLTGPLLLGGVPNLPENFPVSHKDFIGCMRDLHIDGRRVDMAAFVANNGTMAGCQAKLHFCDSGPCKNSGFCSERWGSFSCDCPVGFGGKDCQLTMAHPHHFRGNGTLSWNFGSDMAVSVPWYLGLAFRTRATQGVLMQVQAGPHSTLLCQLDRGLLSVTVTRGSGRASHLLLDQVTVSDGRWHDLRLELQEEPGGRRGHHVLMVSLDFSLFQDTMAVGSELQGLKVKQLHVGGLPPGSAEEAPQGLVGCIQGVWLGSTPSGSPALLPPSHRVNAEPGCVVTNACASGPCPPHADCRDLWQTFSCTCQPGYYGPGCVDACLLNPCQNQGSCRHLPGAPHGYTCDCVGGYFGHHCEHRMDQQCPRGWWGSPTCGPCNCDVHKGFDPNCNKTNGQCHCKEFHYRPRGSDSCLPCDCYPVGSTSRSCAPHSGQCPCRPGALGRQCNSCDSPFAEVTASGCRVLYDACPKSLRSGVWWPQTKFGVLATVPCPRGALGAAVRLCDEAQGWLEPDLFNCTSPAFRELSLLLDGLELNKTALDTMEAKKLAQRLREVTGHTDHYFSQDVRVTARLLAHLLAFESHQQGFGLTATQDAHFNENLLWAGSALLAPETGDLWAALGQRAPGGSPGSAGLVRHLEEYAATLARNMELTYLNPMGLVTPNIMLSIDRMEHPSSPRGARRYPRYHSNLFRGQDAWDPHTHVLLPSQSPRPSPSEVLPTSSSIENSTTSSVVPPPAPPEPEPGISIIILLVYRTLGGLLPAQFQAERRGARLPQNPVMNSPVVSVAVFHGRNFLRGILESPISLEFRLLQTANRSKAICVQWDPPGLAEQHGVWTARDCELVHRNGSHARCRCSRTGTFGVLMDASPRERLEGDLELLAVFTHVVVAVSVAALVLTAAILLSLRSLKSNVRGIHANVAAALGVAELLFLLGIHRTHNQLVCTAVAILLHYFFLSTFAWLFVQGLHLYRMQVEPRNVDRGAMRFYHALGWGVPAVLLGLAVGLDPEGYGNPDFCWISVHEPLIWSFAGPVVLVIVMNGTMFLLAARTSCSTGQREAKKTSALTLRSSFLLLLLVSASWLFGLLAVNHSILAFHYLHAGLCGLQGLAVLLLFCVLNADARAAWMPACLGRKAAPEEARPAPGLGPGAYNNTALFEESGLIRITLGASTVSSVSSARSGRTQDQDSQRGRSYLRDNVLVRHGSAADHTDHSLQAHAGPTDLDVAMFHRDAGADSDSDSDLSLEEERSLSIPSSESEDNGRTRGRFQRPLCRAAQSERLLTHPKDVDGNDLLSYWPALGECEAAPCALQTWGSERRLGLDTSKDAANNNQPDPALTSGDETSLGRAQRQRKGILKNRLQYPLVPQTRGAPELSWCRAATLGHRAVPAASYGRIYAGGGTGSLSQPASRYSSREQLDLLLRRQLSRERLEEAPAPVLRPLSRPGSQECMDAAPGRLEPKDRGSTLPRRQPPRDYPGAMAGRFGSRDALDLGAPREWLSTLPPPRRTRDLDPQPPPLPLSPQRQLSRDPLLPSRPLDSLSRSSNSREQLDQVPSRHPSREALGPLPQLLRAREDSVSGPSHGPSTEQLDILSSILASFNSSALSSVQSSSTPLGPHTTATPSATASVLGPSTPRSATSHSISELSPDSEVPRSEGHS",
# CALY_HUMAN
"MVKLGCSFSGKPGKDPGDQDGAAMDSVPLISPLDISQLQPPLPDQVVIKTQTEYQLSSPDQQNFPDLEGQRLNCSHPEEGRRLPTARMIAFAMALLGCVLIMYKAIWYDQFTCPDGFLLRHKICTPLTLEMYYTEMDPERHRSILAAIGAYPLSRKHGTETPAAWGDGYRAAKEERKGPTQAGAAAAATEPPGKPSAKAEKEAARKAAGSAAPPPAQ",
# PLK2_HUMAN
"MELLRTITYQPAASTKMCEQALGKGCGADSKKKRPPQPPEESQPPQSQAQVPPAAPHHHHHHSHSGPEISRIIVDPTTGKRYCRGKVLGKGGFAKCYEMTDLTNNKVYAAKIIPHSRVAKPHQREKIDKEIELHRILHHKHVVQFYHYFEDKENIYILLEYCSRRSMAHILKARKVLTEPEVRYYLRQIVSGLKYLHEQEILHRDLKLGNFFINEAMELKVGDFGLAARLEPLEHRRRTICGTPNYLSPEVLNKQGHGCESDIWALGCVMYTMLLGRPPFETTNLKETYRCIREARYTMPSSLLAPAKHLIASMLSKNPEDRPSLDDIIRHDFFLQGFTPDRLSSSCCHTVPDFHLSSPAKNFFKKAAAALFGGKKDKARYIDTHNRVSKEDEDIYKLRHDLKKTSITQQPSKHRTDEELQPPTTTVARSGTPAVENKQQIGDAIRMIVRGTLGSCSSSSECLEDSTMGSVADTVARVLRGCLENMPEADCIPKEQLSTSFQWVTKWVDYSNKYGFGYQLSDHTVGVLFNNGAHMSLLPDKKTVHYYAELGQCSVFPATDAPEQFISQVTVLKYFSHYMEENLMDGGDLPSVTDIRRPRLYLLQWLKSDKALMMLFNDGTFQVNFYHDHTKIIICSQNEEYLLTYINEDRISTTFRLTTLLMSGCSSELKNRMEYALNMLLQRCN",
# PEN2_HUMAN
"MNLERVSNEEKLNLCRKYYLGGFAFLPFLWLVNIFWFFREAFLVPAYTEQSQIKGYVWRSAVGFLFWVIVLTSWITIFQIYRPRWGALGDYLSFTIPLGTP",
# NLGN3_HUMAN
"MWLRLGPPSLSLSPKPTVGRSLCLTLWFLSLALRASTQAPAPTVNTHFGKLRGARVPLPSEILGPVDQYLGVPYAAPPIGEKRFLPPEPPPSWSGIRNATHFPPVCPQNIHTAVPEVMLPVWFTANLDIVATYIQEPNEDCLYLNVYVPTEDVKRISKECARKPNKKICRKGGSGAKKQGEDLADNDGDEDEDIRDSGAKPVMVYIHGGSYMEGTGNMIDGSILASYGNVIVITLNYRVGVLGFLSTGDQAAKGNYGLLDQIQALRWVSENIAFFGGDPRRITVFGSGIGASCVSLLTLSHHSEGLFQRAIIQSGSALSSWAVNYQPVKYTSLLADKVGCNVLDTVDMVDCLRQKSAKELVEQDIQPARYHVAFGPVIDGDVIPDDPEILMEQGEFLNYDIMLGVNQGEGLKFVEGVVDPEDGVSGTDFDYSVSNFVDNLYGYPEGKDTLRETIKFMYTDWADRDNPETRRKTLVALFTDHQWVEPSVVTADLHARYGSPTYFYAFYHHCQSLMKPAWSDAAHGDEVPYVFGVPMVGPTDLFPCNFSKNDVMLSAVVMTYWTNFAKTGDPNKPVPQDTKFIHTKANRFEEVAWSKYNPRDQLYLHIGLKPRVRDHYRATKVAFWKHLVPHLYNLHDMFHYTSTTTKVPPPDTTHSSHITRRPNGKTWSTKRPAISPAYSNENAQGSWNGDQDAGPLLVENPRDYSTELSVTIAVGASLLFLNVLAFAALYYRKDKRRQEPLRQPSPQRGAGAPELGAAPEEELAALQLGPTHHECEAGPPHDTLRLTALPDYTLTLRRSPDDIPLMTPNTITMIPNSLVGLQTLHPYNTFAAGFNSTGLPHSHSTTRV",
# IRPL1_HUMAN
"MKAPIPHLILLYATFTQSLKVVTKRGSADGCTDWSIDIKKYQVLVGEPVRIKCALFYGYIRTNYSLAQSAGLSLMWYKSSGPGDFEEPIAFDGSRMSKEEDSIWFRPTLLQDSGLYACVIRNSTYCMKVSISLTVGENDTGLCYNSKMKYFEKAELSKSKEISCRDIEDFLLPTREPEILWYKECRTKTWRPSIVFKRDTLLIREVREDDIGNYTCELKYGGFVVRRTTELTVTAPLTDKPPKLLYPMESKLTIQETQLGDSANLTCRAFFGYSGDVSPLIYWMKGEKFIEDLDENRVWESDIRILKEHLGEQEVSISLIVDSVEEGDLGNYSCYVENGNGRRHASVLLHKRELMYTVELAGGLGAILLLLVCLVTIYKCYKIEIMLFYRNHFGAEELDGDNKDYDAYLSYTKVDPDQWNQETGEEERFALEILPDMLEKHYGYKLFIPDRDLIPTGTYIEDVARCVDQSKRLIIVMTPNYVVRRGWSIFELETRLRNMLVTGEIKVILIECSELRGIMNYQEVEALKHTIKLLTVIKWHGPKCNKLNSKFWKRLQYEMPFKRIEPITHEQALDVSEQGPFGELQTVSAISMAAATSTALATAHPDLRSTFHNTYHSQMRQKHYYRSYEYDVPPTGTLPLTSIGNQHTYCNIPMTLINGQRPQTKSSREQNPDEAHTNSAILPLLPRETSISSVIW",
# SPN90_HUMAN
"MYRALYAFRSAEPNALAFAAGETFLVLERSSAHWWLAARARSGETGYVPPAYLRRLQGLEQDVLQAIDRAIEAVHNTAMRDGGKYSLEQRGVLQKLIHHRKETLSRRGPSASSVAVMTSSTSDHHLDAAAARQPNGVCRAGFERQHSLPSSEHLGADGGLYQIPLPSSQIPPQPRRAAPTTPPPPVKRRDREALMASGSGGHNTMPSGGNSVSSGSSVSSTSLDTLYTSSSPSEPGSSCSPTPPPVPRRGTHTTVSQVQPPPSKASAPEPPAEEEVATGTTSASDDLEALGTLSLGTTEEKAAAEAAVPRTIGAELMELVRRNTGLSHELCRVAIGIIVGHIQASVPASSPVMEQVLLSLVEGKDLSMALPSGQVCHDQQRLEVIFADLARRKDDAQQRSWALYEDEGVIRCYLEELLHILTDADPEVCKKMCKRNEFESVLALVAYYQMEHRASLRLLLLKCFGAMCSLDAAIISTLVSSVLPVELARDMQTDTQDHQKLCYSALILAMVFSMGEAVPYAHYEHLGTPFAQFLLNIVEDGLPLDTTEQLPDLCVNLLLALNLHLPAADQNVIMAALSKHANVKIFSEKLLLLLNRGDDPVRIFKHEPQPPHSVLKFLQDVFGSPATAAIFYHTDMMALIDITVRHIADLSPGDKLRMEYLSLMHAIVRTTPYLQHRHRLPDLQAILRRILNEEETSPQCQMDRMIVREMCKEFLVLGEAPS",
# FLRT3_HUMAN
"MISAAWSIFLIGTKIGLFLQVAPLSVMAKSCPSVCRCDAGFIYCNDRFLTSIPTGIPEDATTLYLQNNQINNAGIPSDLKNLLKVERIYLYHNSLDEFPTNLPKYVKELHLQENNIRTITYDSLSKIPYLEELHLDDNSVSAVSIEEGAFRDSNYLRLLFLSRNHLSTIPWGLPRTIEELRLDDNRISTISSPSLQGLTSLKRLVLDGNLLNNHGLGDKVFFNLVNLTELSLVRNSLTAAPVNLPGTNLRKLYLQDNHINRVPPNAFSYLRQLYRLDMSNNNLSNLPQGIFDDLDNITQLILRNNPWYCGCKMKWVRDWLQSLPVKVNVRGLMCQAPEKVRGMAIKDLNAELFDCKDSGIVSTIQITTAIPNTVYPAQGQWPAPVTKQPDIKNPKLTKDHQTTGSPSRKTITITVKSVTSDTIHISWKLALPMTALRLSWLKLGHSPAFGSITETIVTGERSEYLVTALEPDSPYKVCMVPMETSNLYLFDETPVCIETETAPLRMYNPTTTLNREQEKEPYKNPNLPLAAIIGGAVALVTIALLALVCWYVHRNGSLFSRNCAYSKGRRRKDDYAEAGTKKDNSILEIRETSFQMLPISNEPISKEEFVIHTIFPPNGMNLYKNNHSESSSNRSYRDSGIPDSDHSHS",
# CABP1_HUMAN
"MGGGDGAAFKRPGDGARLQRVLGLGSRREPRSLPAGGPAPRRTAPPPPGHASAGPAAMSSHIAKSESKTSLLKAAAAAASGGSRAPRHGPARDPGLPSRRLPGSCPATPQSSGDPSSRRPLCRPAPREEGARGSQRVLPQAHCRPREALPAAASRPSPSSPLPPARGRDGEERGLSPALGLRGSLRARGRGDSVPAAASEADPFLHRLRPMLSSAFGQDRSLRPEEIEELREAFREFDKDKDGYINCRDLGNCMRTMGYMPTEMELIELSQQINMNLGGHVDFDDFVELMGPKLLAETADMIGVKELRDAFREFDTNGDGEISTSELREAMRKLLGHQVGHRDIEEIIRDVDLNGDGRVDFEEFVRMMSR",
# KCND2_HUMAN
"MAAGVAAWLPFARAAAIGWMPVASGPMPAPPRQERKRTQDALIVLNVSGTRFQTWQDTLERYPDTLLGSSERDFFYHPETQQYFFDRDPDIFRHILNFYRTGKLHYPRHECISAYDEELAFFGLIPEIIGDCCYEEYKDRRRENAERLQDDADTDTAGESALPTMTARQRVWRAFENPHTSTMALVFYYVTGFFIAVSVIANVVETVPCGSSPGHIKELPCGERYAVAFFCLDTACVMIFTVEYLLRLAAAPSRYRFVRSVMSIIDVVAILPYYIGLVMTDNEDVSGAFVTLRVFRVFRIFKFSRHSQGLRILGYTLKSCASELGFLLFSLTMAIIIFATVMFYAEKGSSASKFTSIPAAFWYTIVTMTTLGYGDMVPKTIAGKIFGSICSLSGVLVIALPVPVIVSNFSRIYHQNQRADKRRAQKKARLARIRAAKSGSANAYMQSKRNGLLSNQLQSSEDEQAFVSKSGSSFETQHHHLLHCLEKTTNHEFVDEQVFEESCMEVATVNRPSSHSPSLSSQQGVTSTCCSRRHKKTFRIPNANVSGSHQGSIQELSTIQIRCVERTPLSNSRSSLNAKMEECVKLNCEQPYVTTAIISIPTPPVTTPEGDDRPESPEYSGGNIVRVSAL",
# CRIPT_HUMAN
"MVCEKCEKKLGTVITPDTWKDGARNTTESGGRKLNENKALTSKKARFDPYGKNKFSTCRICKSSVHQPGSHYCQGCAYKKGICAMCGKKVLDTKNYKQTSV",
# ADA22_HUMAN
"MQAAVAVSVPFLLLCVLGTCPPARCGQAGDASLMELEKRKENRFVERQSIVPLRLIYRSGGEDESRHDALDTRVRGDLGGPQLTHVDQASFQVDAFGTSFILDVVLNHDLLSSEYIERHIEHGGKTVEVKGGEHCYYQGHIRGNPDSFVALSTCHGLHGMFYDGNHTYLIEPEENDTTQEDFHFHSVYKSRLFEFSLDDLPSEFQQVNITPSKFILKPRPKRSKRQLRRYPRNVEEETKYIELMIVNDHLMFKKHRLSVVHTNTYAKSVVNMADLIYKDQLKTRIVLVAMETWATDNKFAISENPLITLREFMKYRRDFIKEKSDAVHLFSGSQFESSRSGAAYIGGICSLLKGGGVNEFGKTDLMAVTLAQSLAHNIGIISDKRKLASGECKCEDTWSGCIMGDTGYYLPKKFTQCNIEEYHDFLNSGGGACLFNKPSKLLDPPECGNGFIETGEECDCGTPAECVLEGAECCKKCTLTQDSQCSDGLCCKKCKFQPMGTVCREAVNDCDIRETCSGNSSQCAPNIHKMDGYSCDGVQGICFGGRCKTRDRQCKYIWGQKVTASDKYCYEKLNIEGTEKGNCGKDKDTWIQCNKRDVLCGYLLCTNIGNIPRLGELDGEITSTLVVQQGRTLNCSGGHVKLEEDVDLGYVEDGTPCGPQMMCLEHRCLPVASFNFSTCLSSKEGTICSGNGVCSNELKCVCNRHWIGSDCNTYFPHNDDAKTGITLSGNGVAGTNIIIGIIAGTILVLALILGITAWGYKNYREQRQLPQGDYVKKPGDGDSFYSDIPPGVSTNSASSSKKRSNGLSHSWSERIPDTKHISDICENGRPRSNSWQGNLGGNKKKIRGKRFRPRSNSTETLSPAKSPSSSTGSIASSRKYPYPMPPLPDEDKKVNRQSARLWETSI",
# FRS1L_HUMAN
"MARPPRQHPGVWASLLLLLLTGPAACAASPADDGAGPGGRGPRGRARGDTGADEAVPRHDSSYGTFAGEFYDLRYLSEEGYPFPTAPPVDPFAKIKVDDCGKTKGCFRYGKPGCNAETCDYFLSYRMIGADVEFELSADTDGWVAVGFSSDKKMGGDDVMACVHDDNGRVRIQHFYNVGQWAKEIQRNPARDEEGVFENNRVTCRFKRPVNVPRDETIVDLHLSWYYLFAWGPAIQGSITRHDIDSPPASERVVSIYKYEDIFMPSAAYQTFSSPFCLLLIVALTFYLLMGTP",
# VAPA_HUMAN
"MASASGAMAKHEQILVLDPPTDLKFKGPFTDVVTTNLKLRNPSDRKVCFKVKTTAPRRYCVRPNSGIIDPGSTVTVSVMLQPFDYDPNEKSKHKFMVQTIFAPPNTSDMEAVWKEAKPDELMDSKLRCVFEMPNENDKLNDMEPSKAVPLNASKQDGPMPKPHSVSLNDTETRKLMEECKRLQGEMMKLSEENRHLRDEGLRLRKVAHSDKPGSTSTASFRDNVTSPLPSLLVVIAAIFIGFFLGKFIL",
# MARK1_HUMAN
"MSARTPLPTVNERDTENHTSVDGYTEPHIQPTKSSSRQNIPRCRNSITSATDEQPHIGNYRLQKTIGKGNFAKVKLARHVLTGREVAVKIIDKTQLNPTSLQKLFREVRIMKILNHPNIVKLFEVIETEKTLYLVMEYASGGEVFDYLVAHGRMKEKEARAKFRQIVSAVQYCHQKYIVHRDLKAENLLLDGDMNIKIADFGFSNEFTVGNKLDTFCGSPPYAAPELFQGKKYDGPEVDVWSLGVILYTLVSGSLPFDGQNLKELRERVLRGKYRIPFYMSTDCENLLKKLLVLNPIKRGSLEQIMKDRWMNVGHEEEELKPYTEPDPDFNDTKRIDIMVTMGFARDEINDALINQKYDEVMATYILLGRKPPEFEGGESLSSGNLCQRSRPSSDLNNSTLQSPAHLKVQRSISANQKQRRFSDHAGPSIPPAVSYTKRPQANSVESEQKEEWDKDVARKLGSTTVGSKSEMTASPLVGPERKKSSTIPSNNVYSGGSMARRNTYVCERTTDRYVALQNGKDSSLTEMSVSSISSAGSSVASAVPSARPRHQKSMSTSGHPIKVTLPTIKDGSEAYRPGTTQRVPAASPSAHSISTATPDRTRFPRGSSSRSTFHGEQLRERRSVAYNGPPASPSHETGAFAHARRGTSTGIISKITSKFVRRDPSEGEASGRTDTSRSTSGEPKERDKEEGKDSKPRSLRFTWSMKTTSSMDPNDMMREIRKVLDANNCDYEQKERFLLFCVHGDARQDSLVQWEMEVCKLPRLSLNGVRFKRISGTSIAFKNIASKIANELKL",
# AKA7G_HUMAN
"MERPEAGGINSNECENVSRKKKMSEEFEANTMDSLVDMPFATVDIQDDCGITDEPQINLKRSQENEWVKSDQVKKRKKKRKDYQPNYFLSIPITNKEIIKGIKILQNAIIQQDERLAKAMVSDGSFHITLLVMQLLNEDEVNIGIDALLELKPFIEELLQGKHLTLPFQGIGTFGNQVGFVKLAEGDHVNSLLEIAETANRTFQEKGILVGESRSFKPHLTFMKLSKSPWLRKNGVKKIDPDLYEKFISHRFGEEILYRIDLCSMLKKKQSNGYYHCESSIVIGEKNGGEPDDAELVRLSKRLVENAVLKAVQQYLEETQNKNKPGEGSSVKTEAADQNGNDNENNRK",
# SENP1_HUMAN
"MDDIADRMRMDAGEVTLVNHNSVFKTHLLPQTGFPEDQLSLSDQQILSSRQGHLDRSFTCSTRSAAYNPSYYSDNPSSDSFLGSGDLRTFGQSANGQWRNSTPSSSSSLQKSRNSRSLYLETRKTSSGLSNSFAGKSNHHCHVSAYEKSFPIKPVPSPSWSGSCRRSLLSPKKTQRRHVSTAEETVQEEEREIYRQLLQMVTGKQFTIAKPTTHFPLHLSRCLSSSKNTLKDSLFKNGNSCASQIIGSDTSSSGSASILTNQEQLSHSVYSLSSYTPDVAFGSKDSGTLHHPHHHHSVPHQPDNLAASNTQSEGSDSVILLKVKDSQTPTPSSTFFQAELWIKELTSVYDSRARERLRQIEEQKALALQLQNQRLQEREHSVHDSVELHLRVPLEKEIPVTVVQETQKKGHKLTDSEDEFPEITEEMEKEIKNVFRNGNQDEVLSEAFRLTITRKDIQTLNHLNWLNDEIINFYMNMLMERSKEKGLPSVHAFNTFFFTKLKTAGYQAVKRWTKKVDVFSVDILLVPIHLGVHWCLAVVDFRKKNITYYDSMGGINNEACRILLQYLKQESIDKKRKEFDTNGWQLFSKKSQEIPQQMNGSDCGMFACKYADCITKDRPINFTQQHMPYFRKRMVWEILHRKLL",
# NTRI_HUMAN
"MGVCGYLFLPWKCLVVVSLRLLFLVPTGVPVRSGDATFPKAMDNVTVRQGESATLRCTIDNRVTRVAWLNRSTILYAGNDKWCLDPRVVLLSNTQTQYSIEIQNVDVYDEGPYTCSVQTDNHPKTSRVHLIVQVSPKIVEISSDISINEGNNISLTCIATGRPEPTVTWRHISPKAVGFVSEDEYLEIQGITREQSGDYECSASNDVAAPVVRRVKVTVNYPPYISEAKGTGVPVGQKGTLQCEASAVPSAEFQWYKDDKRLIEGKKGVKVENRPFLSKLIFFNVSEHDYGNYTCVASNKLGHTNASIMLFGPGAVSEVSNGTSRRAGCVWLLPLLVLHLLLKF",
# DLGP2_HUMAN
"MGTAQVLPGILQKHCCILPDRNTESQCTLCGEPEEEEAGDLVQPGISFPGPAEEDLDPQYSWSPTQHFNEERYSPAPRSMKGLSGSRTQPPLCSGHTCGLAPPEDCEHLHHGPDARPPYLLSPADSCPGGRHRCSPRSSVHSECVMMPVVLGDHVSSSTFPRMHYSSHYDTRDDCAVAHAGAKINRIPANLLDQFEKQLPLHRDGFHTLQYQRTSAAAEQRSESPGRIRHLVHSVQKLFTKSHSLEGSSKSNANGTKADGRADDHHHAHHAKHSKRSKSKERKPEGKPRPGMSSWWSSDDNLDSDSTYRTPSVLNRHHLGPVAHCYPDALQSPFGDLSLKTSKSNNDVKCSACEGLALTPDAKYLKRSSWSTLTVSQAKEAYRKSSLNLDKPLLHQDAKPALRPCHYLQVPQDEWGGYPTGGKDEEIPCRRMRSGSYIKAMGDEESGESDSSPKTSPKSAILPEPLLKSIGQRPLGEHQTQTYLQAASDVPVGHSLDPAANYNSPKFRSRNQSYMRAVSTLSQASCVSQVSEAEINGQFESVCESVFSEVESQAMDALDLPGCFRTRSHSYLRAIQAGYSQDDECIPMMTPSDITSTIRSTAAVSYTNYKKTPPPVPPRTTSKPLISVTAQSSTESTQDAYQDSRAQRMSPWPQDSRGLYNSTDSLDSNKAMNLALETAAAQRHLPESQSSSVRTSDKAILVSKAEELLKSRCSSIGIQDSEFPEHQPYPRSDVETATDSDTESRGLREYHSVGVQVEDEKRHGRFKRSNSVTAAVQADLELEGFPGHITTEDKGLQFGSSFQRHSEPSTPTQYSAVRTVRTQGLFSYREDYRTQVDTSTLPPPDPWLEPAIDTVETGRMSPCRRDGSWFLKLLHAETKRMEGWCKEMEREAEENDLSEEILGKIRSAVGSAQLLMSQKFQQFYWLCQQNMDPSAMPRPTSQDLAGYWDMLQLSIEDVSMKFDELQRLRLNDWKMMESPERKEERKVPPPIPKKPPKGKFPITREKSLDLPDRQRQEARRRLMAAKRAASFRQNSASERADSIEIYIPEAQTRL",
# LRFN1_HUMAN
"MAPGPFSSALLSPPPAALPFLLLLWAGASRGQPCPGRCICQNVAPTLTMLCAKTGLLFVPPAIDRRVVELRLTDNFIAAVRRRDFANMTSLVHLTLSRNTIGQVAAGAFADLRALRALHLDSNRLAEVRGDQLRGLGNLRHLILGNNQIRRVESAAFDAFLSTVEDLDLSYNNLEALPWEAVGQMVNLNTLTLDHNLIDHIAEGTFVQLHKLVRLDMTSNRLHKLPPDGLFLRSQGTGPKPPTPLTVSFGGNPLHCNCELLWLRRLTREDDLETCATPEHLTDRYFWSIPEEEFLCEPPLITRQAGGRALVVEGQAVSLRCRAVGDPEPVVHWVAPDGRLLGNSSRTRVRGDGTLDVTITTLRDSGTFTCIASNAAGEATAPVEVCVVPLPLMAPPPAAPPPLTEPGSSDIATPGRPGANDSAAERRLVAAELTSNSVLIRWPAQRPVPGIRMYQVQYNSSVDDSLVYRMIPSTSQTFLVNDLAAGRAYDLCVLAVYDDGATALPATRVVGCVQFTTAGDPAPCRPLRAHFLGGTMIIAIGGVIVASVLVFIVLLMIRYKVYGDGDSRRVKGSRSLPRVSHVCSQTNGAGTGAAQAPALPAQDHYEALREVESQAAPAVAVEAKAMEAETASAEPEVVLGRSLGGSATSLCLLPSEETSGEESRAAVGPRRSRSGALEPPTSAPPTLALVPGGAAARPRPQQRYSFDGDYGALFQSHSYPRRARRTKRHRSTPHLDGAGGGAAGEDGDLGLGSARACLAFTSTEWMLESTV",
# VPS18_HUMAN
"MASILDEYENSLSRSAVLQPGCPSVGIPHSGYVNAQLEKEVPIFTKQRIDFTPSERITSLVVSSNQLCMSLGKDTLLRIDLGKANEPNHVELGRKDDAKVHKMFLDHTGSHLLIALSSTEVLYVNRNGQKVRPLARWKGQLVESVGWNKALGTESSTGPILVGTAQGHIFEAELSASEGGLFGPAPDLYFRPLYVLNEEGGPAPVCSLEAERGPDGRSFVIATTRQRLFQFIGRAAEGAEAQGFSGLFAAYTDHPPPFREFPSNLGYSELAFYTPKLRSAPRAFAWMMGDGVLYGALDCGRPDSLLSEERVWEYPEGVGPGASPPLAIVLTQFHFLLLLADRVEAVCTLTGQVVLRDHFLEKFGPLKHMVKDSSTGQLWAYTERAVFRYHVQREARDVWRTYLDMNRFDLAKEYCRERPDCLDTVLAREADFCFRQRRYLESARCYALTQSYFEEIALKFLEARQEEALAEFLQRKLASLKPAERTQATLLTTWLTELYLSRLGALQGDPEALTLYRETKECFRTFLSSPRHKEWLFASRASIHELLASHGDTEHMVYFAVIMQDYERVVAYHCQHEAYEEALAVLARHRDPQLFYKFSPILIRHIPRQLVDAWIEMGSRLDARQLIPALVNYSQGGEVQQVSQAIRYMEFCVNVLGETEQAIHNYLLSLYARGRPDSLLAYLEQAGASPHRVHYDLKYALRLCAEHGHHRACVHVYKVLELYEEAVDLALQVDVDLAKQCADLPEEDEELRKKLWLKIARHVVQEEEDVQTAMACLASCPLLKIEDVLPFFPDFVTIDHFKEAICSSLKAYNHHIQELQREMEEATASAQRIRRDLQELRGRYGTVEPQDKCATCDFPLLNRPFYLFLCGHMFHADCLLQAVRPGLPAYKQARLEELQRKLGAAPPPAKGSARAKEAEGGAATAGPSREQLKADLDELVAAECVYCGELMIRSIDRPFIDPQRYEEEQLSWL",
# TEN3_HUMAN
"MDVKERRPYCSLTKSRREKERRYTNSSADNEECRVPTQKSYSSSETLKAFDHDSSRLLYGNRVKDLVHREADEFTRQGQNFTLRQLGVCEPATRRGLAFCAEMGLPHRGYSISAGSDADTENEAVMSPEHAMRLWGRGVKSGRSSCLSSRSNSALTLTDTEHENKSDSENEQPASNQGQSTLQPLPPSHKQHSAQHHPSITSLNRNSLTNRRNQSPAPPAALPAELQTTPESVQLQDSWVLGSNVPLESRHFLFKTGTGTTPLFSTATPGYTMASGSVYSPPTRPLPRNTLSRSAFKFKKSSKYCSWKCTALCAVGVSVLLAILLSYFIAMHLFGLNWQLQQTENDTFENGKVNSDTMPTNTVSLPSGDNGKLGGFTQENNTIDSGELDIGRRAIQEIPPGIFWRSQLFIDQPQFLKFNISLQKDALIGVYGRKGLPPSHTQYDFVELLDGSRLIAREQRSLLETERAGRQARSVSLHEAGFIQYLDSGIWHLAFYNDGKNAEQVSFNTIVIESVVECPRNCHGNGECVSGTCHCFPGFLGPDCSRAACPVLCSGNGQYSKGRCLCFSGWKGTECDVPTTQCIDPQCGGRGICIMGSCACNSGYKGESCEEADCIDPGCSNHGVCIHGECHCSPGWGGSNCEILKTMCPDQCSGHGTYLQESGSCTCDPNWTGPDCSNEICSVDCGSHGVCMGGTCRCEEGWTGPACNQRACHPRCAEHGTCKDGKCECSQGWNGEHCTIEGCPGLCNSNGRCTLDQNGWHCVCQPGWRGAGCDVAMETLCTDSKDNEGDGLIDCMDPDCCLQSSCQNQPYCRGLPDPQDIISQSLQSPSQQAAKSFYDRISFLIGSDSTHVIPGESPFNKSLASVIRGQVLTADGTPLIGVNVSFFHYPEYGYTITRQDGMFDLVANGGASLTLVFERSPFLTQYHTVWIPWNVFYVMDTLVMKKEENDIPSCDLSGFVRPNPIIVSSPLSTFFRSSPEDSPIIPETQVLHEETTIPGTDLKLSYLSSRAAGYKSVLKITMTQSIIPFNLMKVHLMVAVVGRLFQKWFPASPNLAYTFIWDKTDAYNQKVYGLSEAVVSVGYEYESCLDLTLWEKRTAILQGYELDASNMGGWTLDKHHVLDVQNGILYKGNGENQFISQQPPVVSSIMGNGRRRSISCPSCNGQADGNKLLAPVALACGIDGSLYVGDFNYVRRIFPSGNVTSVLELSSNPAHRYYLATDPVTGDLYVSDTNTRRIYRPKSLTGAKDLTKNAEVVAGTGEQCLPFDEARCGDGGKAVEATLMSPKGMAVDKNGLIYFVDGTMIRKVDQNGIISTLLGSNDLTSARPLTCDTSMHISQVRLEWPTDLAINPMDNSIYVLDNNVVLQITENRQVRIAAGRPMHCQVPGVEYPVGKHAVQTTLESATAIAVSYSGVLYITETDEKKINRIRQVTTDGEISLVAGIPSECDCKNDANCDCYQSGDGYAKDAKLSAPSSLAASPDGTLYIADLGNIRIRAVSKNKPLLNSMNFYEVASPTDQELYIFDINGTHQYTVSLVTGDYLYNFSYSNDNDITAVTDSNGNTLRIRRDPNRMPVRVVSPDNQVIWLTIGTNGCLKSMTAQGLELVLFTYHGNSGLLATKSDETGWTTFFDYDSEGRLTNVTFPTGVVTNLHGDMDKAITVDIESSSREEDVSITSNLSSIDSFYTMVQDQLRNSYQIGYDGSLRIIYASGLDSHYQTEPHVLAGTANPTVAKRNMTLPGENGQNLVEWRFRKEQAQGKVNVFGRKLRVNGRNLLSVDFDRTTKTEKIYDDHRKFLLRIAYDTSGHPTLWLPSSKLMAVNVTYSSTGQIASIQRGTTSEKVDYDGQGRIVSRVFADGKTWSYTYLEKSMVLLLHSQRQYIFEYDMWDRLSAITMPSVARHTMQTIRSIGYYRNIYNPPESNASIITDYNEEGLLLQTAFLGTSRRVLFKYRRQTRLSEILYDSTRVSFTYDETAGVLKTVNLQSDGFICTIRYRQIGPLIDRQIFRFSEDGMVNARFDYSYDNSFRVTSMQGVINETPLPIDLYQFDDISGKVEQFGKFGVIYYDINQIISTAVMTYTKHFDAHGRIKEIQYEIFRSLMYWITIQYDNMGRVTKREIKIGPFANTTKYAYEYDVDGQLQTVYLNEKIMWRYNYDLNGNLHLLNPSNSARLTPLRYDLRDRITRLGDVQYRLDEDGFLRQRGTEIFEYSSKGLLTRVYSKGSGWTVIYRYDGLGRRVSSKTSLGQHLQFFYADLTYPTRITHVYNHSSSEITSLYYDLQGHLFAMEISSGDEFYIASDNTGTPLAVFSSNGLMLKQIQYTAYGEIYFDSNIDFQLVIGFHGGLYDPLTKLIHFGERDYDILAGRWTTPDIEIWKRIGKDPAPFNLYMFRNNNPASKIHDVKDYITDVNSWLVTFGFHLHNAIPGFPVPKFDLTEPSYELVKSQQWDDIPPIFGVQQQVARQAKAFLSLGKMAEVQVSRRRAGGAQSWLWFATVKSLIGKGVMLAVSQGRVQTNVLNIANEDCIKVAAVLNNAFYLENLHFTIEGKDTHYFIKTTTPESDLGTLRLTSGRKALENGINVTVSQSTTVVNGRTRRFADVEMQFGALALHVRYGMTLDEEKARILEQARQRALARAWAREQQRVRDGEEGARLWTEGEKRQLLSAGKVQGYDGYYVLSVEQYPELADSANNIQFLRQSEIGRR",
# PAK5_HUMAN
"MFGKKKKKIEISGPSNFEHRVHTGFDPQEQKFTGLPQQWHSLLADTANRPKPMVDPSCITPIQLAPMKTIVRGNKPCKETSINGLLEDFDNISVTRSNSLRKESPPTPDQGASSHGPGHAEENGFITFSQYSSESDTTADYTTEKYREKSLYGDDLDPYYRGSHAAKQNGHVMKMKHGEAYYSEVKPLKSDFARFSADYHSHLDSLSKPSEYSDLKWEYQRASSSSPLDYSFQFTPSRTAGTSGCSKESLAYSESEWGPSLDDYDRRPKSSYLNQTSPQPTMRQRSRSGSGLQEPMMPFGASAFKTHPQGHSYNSYTYPRLSEPTMCIPKVDYDRAQMVLSPPLSGSDTYPRGPAKLPQSQSKSGYSSSSHQYPSGYHKATLYHHPSLQSSSQYISTASYLSSLSLSSSTYPPPSWGSSSDQQPSRVSHEQFRAALQLVVSPGDPREYLANFIKIGEGSTGIVCIATEKHTGKQVAVKKMDLRKQQRRELLFNEVVIMRDYHHDNVVDMYSSYLVGDELWVVMEFLEGGALTDIVTHTRMNEEQIATVCLSVLRALSYLHNQGVIHRDIKSDSILLTSDGRIKLSDFGFCAQVSKEVPKRKSLVGTPYWMAPEVISRLPYGTEVDIWSLGIMVIEMIDGEPPYFNEPPLQAMRRIRDSLPPRVKDLHKVSSVLRGFLDLMLVREPSQRATAQELLGHPFLKLAGPPSCIVPLMRQYRHH",
# ABI3_HUMAN
"MAELQQLQEFEIPTGREALRGNHSALLRVADYCEDNYVQATDKRKALEETMAFTTQALASVAYQVGNLAGHTLRMLDLQGAALRQVEARVSTLGQMVNMHMEKVARREIGTLATVQRLPPGQKVIAPENLPPLTPYCRRPLNFGCLDDIGHGIKDLSTQLSRTGTLSRKSIKAPATPASATLGRPPRIPEPVHLPVVPDGRLSAASSAFSLASAGSAEGVGGAPTPKGQAAPPAPPLPSSLDPPPPPAAVEVFQRPPTLEELSPPPPDEELPLPLDLPPPPPLDGDELGLPPPPPGFGPDEPSWVPASYLEKVVTLYPYTSQKDNELSFSEGTVICVTRRYSDGWCEGVSSEGTGFFPGNYVEPSC",
# KIF17_HUMAN
"MASEAVKVVVRCRPMNQRERELRCQPVVTVDCARAQCCIQNPGAADEPPKQFTFDGAYHVDHVTEQIYNEIAYPLVEGVTEGYNGTIFAYGQTGSGKSFTMQGLPDPPSQRGIIPRAFEHVFESVQCAENTKFLVRASYLEIYNEDVRDLLGADTKQKLELKEHPEKGVYVKGLSMHTVHSVAQCEHIMETGWKNRSVGYTLMNKDSSRSHSIFTISIEMSAVDERGKDHLRAGKLNLVDLAGSERQSKTGATGERLKEATKINLSLSALGNVISALVDGRCKHVPYRDSKLTRLLQDSLGGNTKTLMVACLSPADNNYDETLSTLRYANRAKNIRNKPRINEDPKDALLREYQEEIKKLKAILTQQMSPSSLSALLSRQVPPDPVQVEEKLLPQPVIQHDVEAEKQLIREEYEERLARLKADYKAEQESRARLEEDITAMRNSYDVRLSTLEENLRKETEAVLQVGVLYKAEVMSRAEFASSAEYPPAFQYETVVKPKVFSTTDTLPSDDVSKTQVSSRFAELPKVEPSKSEISLGSSESSSLEETSVSEAFPGPEEPSNVEVSMPTEESRSRYFLDECLGQEAAGHLLGEQNYLPQEEPQEVPLQGLLGLQDPFAEVEAKLARLSSTVARTDAPQADVPKVPVQVPAPTDLLEPSDARPEAEAADDFPPRPEVDLASEVALEVVRTAEPGVWLEAQAPVALVAQPEPLPATAGVKRESVGMEVAVLTDDPLPVVDQQQVLARLQLLEQQVVGGEQAKNKDLKEKHKRRKRYADERRKQLVAALQNSDEDSGDWVLLNVYDSIQEEVRAKSKLLEKMQRKLRAAEVEIKDLQSEFQLEKIDYLATIRRQERDSMLLQQLLEQVQPLIRRDCNYSNLEKILRESCWDEDNGFWKIPHPVITKTSLPVAVSTGPQNKPARKTSAADNGEPNMEDDRYRLMLSRSNSENIASNYFRSKRASQILSTDARKSLTHHNSPPGLSCPLSNNSAIPPTQAPEMPQPRPFRLESLDIPFTKAKRKKSKSNFGSEPL",
# PCD10_HUMAN
"MIVLLLFALLWMVEGVFSQLHYTVQEEQEHGTFVGNIAEDLGLDITKLSARGFQTVPNSRTPYLDLNLETGVLYVNEKIDREQICKQSPSCVLHLEVFLENPLELFQVEIEVLDINDNPPSFPEPDLTVEISESATPGTRFPLESAFDPDVGTNSLRDYEITPNSYFSLDVQTQGDGNRFAELVLEKPLDREQQAVHRYVLTAVDGGGGGGVGEGGGGGGGAGLPPQQQRTGTALLTIRVLDSNDNVPAFDQPVYTVSLPENSPPGTLVIQLNATDPDEGQNGEVVYSFSSHISPRARELFGLSPRTGRLEVSGELDYEESPVYQVYVQAKDLGPNAVPAHCKVLVRVLDANDNAPEISFSTVKEAVSEGAAPGTVVALFSVTDRDSEENGQVQCELLGDVPFRLKSSFKNYYTIVTEAPLDREAGDSYTLTVVARDRGEPALSTSKSIQVQVSDVNDNAPRFSQPVYDVYVTENNVPGAYIYAVSATDRDEGANAQLAYSILECQIQGMSVFTYVSINSENGYLYALRSFDYEQLKDFSFQVEARDAGSPQALAGNATVNILIVDQNDNAPAIVAPLPGRNGTPAREVLPRSAEPGYLLTRVAAVDADDGENARLTYSIVRGNEMNLFRMDWRTGELRTARRVPAKRDPQRPYELVIEVRDHGQPPLSSTATLVVQLVDGAVEPQGGGGSGGGGSGEHQRPSRSGGGETSLDLTLILIIALGSVSFIFLLAMIVLAVRCQKEKKLNIYTCLASDCCLCCCCCGGGGSTCCGRQARARKKKLSKSDIMLVQSSNVPSNPAQVPIEESGGFGSHHHNQNYCYQVCLTPESAKTDLMFLKPCSPSRSTDTEHNPCGAIVTGYTDQQPDIISNGSILSNETKHQRAELSYLVDRPRRVNSSAFQEADIVSSKDSGHGDSEQGDSDHDATNRAQSAGMDLFSNCTEECKALGHSDRCWMPSFVPSDGRQAADYRSNLHVPGMDSVPDTEVFETPEAQPGAERSFSTFGKEKALHSTLERKELDGLLTNTRAPYKPPYLTRKRIC",
# SI1L2_HUMAN
"MSDPRQSQEEKHKLGRASSKFKDPPRIMQSDDYFARKFKAINGNMGPTTSLNASNSNETGGGGPANGTPAVPKMGVRARVSEWPPKKDCSKELTCKALWESRSQTSYESITSVLQNGQSDQSEGQQDEQLDLDFVEAKYTIGDIFVHSPQRGLHPIRQRSNSDVTISDIDAEDVLDQNAVNPNTGAALHREYGSTSSIDRQGLSGENFFAMLRGYRVENYDHKAMVPFGFPEFFRCDPAISPSLHAAAQISRGEFVRISGLDYVDSALLMGRDRDKPFKRRLKSESVETSLFRKLRTVKSEHETFKFTSELEESRLERGIRPWNCQRCFAHYDVQSILFNINEAMATRANVGKRKNITTGASAASQTQMPTGQTGNCESPLGSKEDLNSKENLDADEGDGKSNDLVLSCPYFRNETGGEGDRRIALSRANSSSFSSGESCSFESSLSSHCTNAGVSVLEVPRENQPIHREKVKRYIIEHIDLGAYYYRKFFYGKEHQNYFGIDENLGPVAVSIRREKVEDAKEKEGSQFNYRVAFRTSELTTLRGAILEDAIPSTARHGTARGLPLKEVLEYVIPELSIQCLRQASNSPKVSEQLLKLDEQGLSFQHKIGILYCKAGQSTEEEMYNNETAGPAFEEFLDLLGQRVRLKGFSKYRAQLDNKTDSTGTHSLYTTYKDYELMFHVSTLLPYMPNNRQQLLRKRHIGNDIVTIVFQEPGALPFTPKSIRSHFQHVFVIVKVHNPCTENVCYSVGVSRSKDVPPFGPPIPKGVTFPKSAVFRDFLLAKVINAENAAHKSEKFRAMATRTRQEYLKDLAENFVTTATVDTSVKFSFITLGAKKKEKVKPRKDAHLFSIGAIMWHVIARDFGQSADIECLLGISNEFIMLIEKDSKNVVFNCSCRDVIGWTSGLVSIKVFYERGECVLLSSVDNCAEDIREIVQRLVIVTRGCETVEMTLRRNGLGQLGFHVNFEGIVADVEPFGFAWKAGLRQGSRLVEICKVAVATLTHEQMIDLLRTSVTVKVVIIQPHDDGSPRRGCSELCRIPMVEYKLDSEGTPCEYKTPFRRNTTWHRVPTPALQPLSRASPIPGTPDRLPCQQLLQQAQAAIPRSTSFDRKLPDGTRSSPSNQSSSSDPGPGGSGPWRPQVGYDGCQSPLLLEHQGSGPLECDGAREREDTMEASRHPETKWHGPPSKVLGSYKERALQKDGSCKDSPNKLSHIGDKSCSSHSSSNTLSSNTSSNSDDKHFGSGDLMDPELLGLTYIKGASTDSGIDTAPCMPATILGPVHLAGSRSLIHSRAEQWADAADVSGPDDEPAKLYSVHGYASTISAGSAAEGSMGDLSEISSHSSGSHHSGSPSAHCSKSSGSLDSSKVYIVSHSSGQQVPGSMSKPYHRQGAVNKYVIGWKKSEGSPPPEEPEVTECPGMYSEMDVMSTATQHQTVVGDAVAETQHVLSKEDFLKLMLPDSPLVEEGRRKFSFYGNLSPRRSLYRTLSDESICSNRRGSSFGSSRSSVLDQALPNDILFSTTPPYHSTLPPRAHPAPSMGSLRNEFWFSDGSLSDKSKCADPGLMPLPDTATGLDWTHLVDAARAFEGLDSDEELGLLCHHTSYLDQRVASFCTLTDMQHGQDLEGAQELPLCVDPGSGKEFMDTTGERSPSPLTGKVNQLELILRQLQTDLRKEKQDKAVLQAEVQHLRQDNMRLQEESQTATAQLRKFTEWFFTTIDKKS",
# CPSF2_HUMAN
"MTSIIKLTTLSGVQEESALCYLLQVDEFRFLLDCGWDEHFSMDIIDSLRKHVHQIDAVLLSHPDPLHLGALPYAVGKLGLNCAIYATIPVYKMGQMFMYDLYQSRHNTEDFTLFTLDDVDAAFDKIQQLKFSQIVNLKGKGHGLSITPLPAGHMIGGTIWKIVKDGEEEIVYAVDFNHKREIHLNGCSLEMLSRPSLLITDSFNATYVQPRRKQRDEQLLTNVLETLRGDGNVLIAVDTAGRVLELAQLLDQIWRTKDAGLGVYSLALLNNVSYNVVEFSKSQVEWMSDKLMRCFEDKRNNPFQFRHLSLCHGLSDLARVPSPKVVLASQPDLECGFSRDLFIQWCQDPKNSIILTYRTTPGTLARFLIDNPSEKITEIELRKRVKLEGKELEEYLEKEKLKKEAAKKLEQSKEADIDSSDESDIEEDIDQPSAHKTKHDLMMKGEGSRKGSFFKQAKKSYPMFPAPEERIKWDEYGEIIKPEDFLVPELQATEEEKSKLESGLTNGDEPMDQDLSDVPTKCISTTESIEIKARVTYIDYEGRSDGDSIKKIINQMKPRQLIIVHGPPEASQDLAECCRAFGGKDIKVYMPKLHETVDATSETHIYQVRLKDSLVSSLQFCKAKDAELAWIDGVLDMRVSKVDTGVILEEGELKDDGEDSEMQVEAPSDSSVIAQQKAMKSLFGDDEKETGEESEIIPTLEPLPPHEVPGHQSVFMNEPRLSDFKQVLLREGIQAEFVGGVLVCNNQVAVRRTETGRIGLEGCLCQDFYRIRDLLYEQYAIV",
# TUTLA_HUMAN
"MVWCLGLAVLSLVISQGADGRGKPEVVSVVGRAGESVVLGCDLLPPAGRPPLHVIEWLRFGFLLPIFIQFGLYSPRIDPDYVGRVRLQKGASLQIEGLRVEDQGWYECRVFFLDQHIPEDDFANGSWVHLTVNSPPQFQETPPAVLEVQELEPVTLRCVARGSPLPHVTWKLRGKDLGQGQGQVQVQNGTLRIRRVERGSSGVYTCQASSTEGSATHATQLLVLGPPVIVVPPKNSTVNASQDVSLACHAEAYPANLTYSWFQDNINVFHISRLQPRVRILVDGSLRLLATQPDDAGCYTCVPSNGLLHPPSASAYLTVLYPAQVTAMPPETPLPIGMPGVIRCPVRANPPLLFVSWTKDGKALQLDKFPGWSQGTEGSLIIALGNEDALGEYSCTPYNSLGTAGPSPVTRVLLKAPPAFIERPKEEYFQEVGRELLIPCSAQGDPPPVVSWTKVGRGLQGQAQVDSNSSLILRPLTKEAHGHWECSASNAVARVATSTNVYVLGTSPHVVTNVSVVALPKGANVSWEPGFDGGYLQRFSVWYTPLAKRPDRMHHDWVSLAVPVGAAHLLVPGLQPHTQYQFSVLAQNKLGSGPFSEIVLSAPEGLPTTPAAPGLPPTEIPPPLSPPRGLVAVRTPRGVLLHWDPPELVPKRLDGYVLEGRQGSQGWEVLDPAVAGTETELLVPGLIKDVLYEFRLVAFAGSFVSDPSNTANVSTSGLEVYPSRTQLPGLLPQPVLAGVVGGVCFLGVAVLVSILAGCLLNRRRAARRRRKRLRQDPPLIFSPTGKSAAPSALGSGSPDSVAKLKLQGSPVPSLRQSLLWGDPAGTPSPHPDPPSSRGPLPLEPICRGPDGRFVMGPTVAAPQERSGREQAEPRTPAQRLARSFDCSSSSPSGAPQPLCIEDISPVAPPPAAPPSPLPGPGPLLQYLSLPFFREMNVDGDWPPLEEPSPAAPPDYMDTRRCPTSSFLRSPETPPVSPRESLPGAVVGAGATAEPPYTALADWTLRERLLPGLLPAAPRGSLTSQSSGRGSASFLRPPSTAPSAGGSYLSPAPGDTSSWASGPERWPRREHVVTVSKRRNTSVDENYEWDSEFPGDMELLETLHLGLASSRLRPEAEPELGVKTPEEGCLLNTAHVTGPEARCAALREEFLAFRRRRDATRARLPAYRQPVPHPEQATLL",
# NRX2A_HUMAN
"MASGSRWRPTPPPLLLLLLLALAARADGLEFGGGPGQWARYARWAGAASSGELSFSLRTNATRALLLYLDDGGDCDFLELLLVDGRLRLRFTLSCAEPATLQLDTPVADDRWHMVLLTRDARRTALAVDGEARAAEVRSKRREMQVASDLFVGGIPPDVRLSALTLSTVKYEPPFRGLLANLKLGERPPALLGSQGLRGATADPLCAPARNPCANGGLCTVLAPGEVGCDCSHTGFGGKFCSEEEHPMEGPAHLTLNSEVGSLLFSEGGAGRGGAGDVHQPTKGKEEFVATFKGNEFFCYDLSHNPIQSSTDEITLAFRTLQRNGLMLHTGKSADYVNLSLKSGAVWLVINLGSGAFEALVEPVNGKFNDNAWHDVRVTRNLRQHAGIGHAMVNKLHYLVTISVDGILTTTGYTQEDYTMLGSDDFFYIGGSPNTADLPGSPVSNNFMGCLKDVVYKNNDFKLELSRLAKEGDPKMKLQGDLSFRCEDVAALDPVTFESPEAFVALPRWSAKRTGSISLDFRTTEPNGLLLFSQGRRAGGGAGSHSSAQRADYFAMELLDGHLYLLLDMGSGGIKLRASSRKVNDGEWCHVDFQRDGRKGSISVNSRSTPFLATGDSEILDLESELYLGGLPEGGRVDLPLPPEVWTAALRAGYVGCVRDLFIDGRSRDLRGLAEAQGAVGVAPFCSRETLKQCASAPCRNGGVCREGWNRFICDCIGTGFLGRVCEREATVLSYDGSMYMKIMLPNAMHTEAEDVSLRFMSQRAYGLMMATTSRESADTLRLELDGGQMKLTVNLDCLRVGCAPSKGPETLFAGHKLNDNEWHTVRVVRRGKSLQLSVDNVTVEGQMAGAHMRLEFHNIETGIMTERRFISVVPSNFIGHLSGLVFNGQPYMDQCKDGDITYCELNARFGLRAIVADPVTFKSRSSYLALATLQAYASMHLFFQFKTTAPDGLLLFNSGNGNDFIVIELVKGYIHYVFDLGNGPSLMKGNSDKPVNDNQWHNVVVSRDPGNVHTLKIDSRTVTQHSNGARNLDLKGELYIGGLSKNMFSNLPKLVASRDGFQGCLASVDLNGRLPDLIADALHRIGQVERGCDGPSTTCTEESCANQGVCLQQWDGFTCDCTMTSYGGPVCNDPGTTYIFGKGGALITYTWPPNDRPSTRMDRLAVGFSTHQRSAVLVRVDSASGLGDYLQLHIDQGTVGVIFNVGTDDITIDEPNAIVSDGKYHVVRFTRSGGNATLQVDSWPVNERYPAGNFDNERLAIARQRIPYRLGRVVDEWLLDKGRQLTIFNSQAAIKIGGRDQGRPFQGQVSGLYYNGLKVLALAAESDPNVRTEGHLRLVGEGPSVLLSAETTATTLLADMATTIMETTTTMATTTTRRGRSPTLRDSTTQNTDDLLVASAECPSDDEDLEECEPSTGGELILPIITEDSLDPPPVATRSPFVPPPPTFYPFLTGVGATQDTLPPPAARRPPSGGPCQAERDDSDCEEPIEASGFASGEVFDSSLPPTDDEDFYTTFPLVTDRTTLLSPRKPAPRPNLRTDGATGAPGVLFAPSAPAPNLPAGKMNHRDPLQPLLENPPLGPGAPTSFEPRRPPPLRPGVTSAPGFPHLPTANPTGPGERGPPGAVEVIRESSSTTGMVVGIVAAAALCILILLYAMYKYRNRDEGSYQVDQSRNYISNSAQSNGAVVKEKAPAAPKTPSKAKKNKDKEYYV",
# VGLU1_HUMAN
"MEFRQEEFRKLAGRALGKLHRLLEKRQEGAETLELSADGRPVTTQTRDPPVVDCTCFGLPRRYIIAIMSGLGFCISFGIRCNLGVAIVSMVNNSTTHRGGHVVVQKAQFSWDPETVGLIHGSFFWGYIVTQIPGGFICQKFAANRVFGFAIVATSTLNMLIPSAARVHYGCVIFVRILQGLVEGVTYPACHGIWSKWAPPLERSRLATTAFCGSYAGAVVAMPLAGVLVQYSGWSSVFYVYGSFGIFWYLFWLLVSYESPALHPSISEEERKYIEDAIGESAKLMNPLTKFSTPWRRFFTSMPVYAIIVANFCRSWTFYLLLISQPAYFEEVFGFEISKVGLVSALPHLVMTIIVPIGGQIADFLRSRRIMSTTNVRKLMNCGGFGMEATLLLVVGYSHSKGVAISFLVLAVGFSGFAISGFNVNHLDIAPRYASILMGISNGVGTLSGMVCPIIVGAMTKHKTREEWQYVFLIASLVHYGGVIFYGVFASGEKQPWAEPEEMSEEKCGFVGHDQLAGSDDSEMEDEAEPPGAPPAPPPSYGATHSTFQPPRPPPPVRDY",
# VGLU2_HUMAN
"MESVKQRILAPGKEGLKNFAGKSLGQIYRVLEKKQDTGETIELTEDGKPLEVPERKAPLCDCTCFGLPRRYIIAIMSGLGFCISFGIRCNLGVAIVDMVNNSTIHRGGKVIKEKAKFNWDPETVGMIHGSFFWGYIITQIPGGYIASRLAANRVFGAAILLTSTLNMLIPSAARVHYGCVIFVRILQGLVEGVTYPACHGIWSKWAPPLERSRLATTSFCGSYAGAVIAMPLAGILVQYTGWSSVFYVYGSFGMVWYMFWLLVSYESPAKHPTITDEERRYIEESIGESANLLGAMEKFKTPWRKFFTSMPVYAIIVANFCRSWTFYLLLISQPAYFEEVFGFEISKVGMLSAVPHLVMTIIVPIGGQIADFLRSKQILSTTTVRKIMNCGGFGMEATLLLVVGYSHTRGVAISFLVLAVGFSGFAISGFNVNHLDIAPRYASILMGISNGVGTLSGMVCPIIVGAMTKNKSREEWQYVFLIAALVHYGGVIFYAIFASGEKQPWADPEETSEEKCGFIHEDELDEETGDITQNYINYGTTKSYGATTQANGGWPSGWEKKEEFVQGEVQDSHSYKDRVDYS",
# GBG13_HUMAN
"MEEWDVPQMKKEVESLKYQLAFQREMASKTIPELLKWIEDGIPKDPFLNPDLMKNNPWVEKGKCTIL",
# NCDN_HUMAN
"MSCCDLAAAGQLGKASIMASDCEPALNQAEGRNPTLERYLGALREAKNDSEQFAALLLVTKAVKAGDIDAKTRRRIFDAVGFTFPNRLLTTKEAPDGCPDHVLRALGVALLACFCSDPELAAHPQVLNKIPILSTFLTARGDPDDAARRSMIDDTYQCLTAVAGTPRGPRHLIAGGTVSALCQAYLGHGYGFDQALALLVGLLAAAETQCWKEAEPDLLAVLRGLSEDFQKAEDASKFELCQLLPLFLPPTTVPPECYRDLQAGLARILGSKLSSWQRNPALKLAARLAHACGSDWIPAGSSGSKFLALLVNLACVEVRLALEETGTEVKEDVVTACYALMELGIQECTRCEQSLLKEPQKVQLVSVMKEAIGAVIHYLLQVGSEKQKEPFVFASVRILGAWLAEETSSLRKEVCQLLPFLVRYAKTLYEEAEEANDLSQQVANLAISPTTPGPTWPGDALRLLLPGWCHLTVEDGPREILIKEGAPSLLCKYFLQQWELTSPGHDTSVLPDSVEIGLQTCCHIFLNLVVTAPGLIKRDACFTSLMNTLMTSLPALVQQQGRLLLAANVATLGLLMARLLSTSPALQGTPASRGFFAAAILFLSQSHVARATPGSDQAVLALSPEYEGIWADLQELWFLGMQAFTGCVPLLPWLAPAALRSRWPQELLQLLGSVSPNSVKPEMVAAYQGVLVELARANRLCREAMRLQAGEETASHYRMAALEQCLSEP",
# EP15R_HUMAN
"MAAPLIPLSQQIPTGNSLYESYYKQVDPAYTGRVGASEAALFLKKSGLSDIILGKIWDLADPEGKGFLDKQGFYVALRLVACAQSGHEVTLSNLNLSMPPPKFHDTSSPLMVTPPSAEAHWAVRVEEKAKFDGIFESLLPINGLLSGDKVKPVLMNSKLPLDVLGRVWDLSDIDKDGHLDRDEFAVAMHLVYRALEKEPVPSALPPSLIPPSKRKKTVFPGAVPVLPASPPPKDSLRSTPSHGSVSSLNSTGSLSPKHSLKQTQPTVNWVVPVADKMRFDEIFLKTDLDLDGYVSGQEVKEIFMHSGLTQNLLAHIWALADTRQTGKLSKDQFALAMYFIQQKVSKGIDPPQVLSPDMVPPSERGTPGPDSSGSLGSGEFTGVKELDDISQEIAQLQREKYSLEQDIREKEEAIRQKTSEVQELQNDLDRETSSLQELEAQKQDAQDRLDEMDQQKAKLRDMLSDVRQKCQDETQMISSLKTQIQSQESDLKSQEDDLNRAKSELNRLQQEETQLEQSIQAGRVQLETIIKSLKSTQDEINQARSKLSQLHESRQEAHRSLEQYDQVLDGAHGASLTDLANLSEGVSLAERGSFGAMDDPFKNKALLFSNNTQELHPDPFQTEDPFKSDPFKGADPFKGDPFQNDPFAEQQTTSTDPFGGDPFKESDPFRGSATDDFFKKQTKNDPFTSDPFTKNPSLPSKLDPFESSDPFSSSSVSSKGSDPFGTLDPFGSGSFNSAEGFADFSQMSKPPPSGPFTSSLGGAGFSDDPFKSKQDTPALPPKKPAPPRPKPPSGKSTPVSQLGSADFPEAPDPFQPLGADSGDPFQSKKGFGDPFSGKDPFVPSSAAKPSKASASGFADFTSVS",
# GBG12_HUMAN
"MSSKTASTNNIAQARRTVQQLRLEASIERIKVSKASADLMSYCEEHARSDPLLIGIPTSENPFKDKKTCIIL",
# CPNE7_HUMAN
"MSAGSERGAAATPGGLPAPCASKVELRLSCRHLLDRDPLTKSDPSVALLQQAQGQWVQVGRTEVVRSSLHPVFSKVFTVDYYFEEVQRLRFEVYDTHGPSGFSCQEDDFLGGMECTLGQPAQKWLLQVVMRVSVDVLGPAGHCAKHFLCCTESSHLARTGPSFLLRYDDLCLPWATAGAVRWWTCRGGHTQGWQIVAQKKVTRPLLLKFGRNAGKSTITVIAEDISGNNGYVELSFRARKLDDKDLFSKSDPFLELYRVNDDQGLQLVYRTEVVKNNLNPVWEAFKVSLSSLCSCEETRPLKCLVWDYDSRGKHDFIGEFSTTFEEMQKAFEEGQAQWDCVNPKYKQKRRSYKNSGVVVLADLKFHRVYSFLDYIMGGCQIHFTVAIDFTASNGDPRNSCSLHYINPYQPNEYLKALVSVGEICQDYDSDKRFSALGFGARIPPKYEVSHDFAINFNPEDDECEGIQGVVEAYQNCLPRVQLYGPTNVAPIISKVARVAAAEESTGKASQYYILLILTDGVVTDMADTREAIVRASRLPMSIIIVGVGNADFTDMQVLDGDDGVLRSPRGEPALRDIVQFVPFRELKNASPAALAKCVLAEVPKQVVEYYSHRGLPPRSLGVPAGEASPGCTP",
# P2RX2_HUMAN
"MAAAQPKYPAGATARRLARGCWSALWDYETPKVIVVRNRRLGVLYRAVQLLILLYFVWYVFIVQKSYQESETGPESSIITKVKGITTSEHKVWDVEEYVKPPEGGSVFSIITRVEATHSQTQGTCPESIRVHNATCLSDADCVAGELDMLGNGLRTGRCVPYYQGPSKTCEVFGWCPVEDGASVSQFLGTMAPNFTILIKNSIHYPKFHFSKGNIADRTDGYLKRCTFHEASDLYCPIFKLGFIVEKAGESFTELAHKGGVIGVIINWDCDLDLPASECNPKYSFRRLDPKHVPASSGYNFRFAKYYKINGTTTRTLIKAYGIRIDVIVHGQAGKFSLIPTIINLATALTSVGVGSFLCDWILLTFMNKNKVYSHKKFDKVCTPSHPSGSWPVTLARVLGQAPPEPGHRSEDQHPSPPSGQEGQQGAECGPAFPPLRPCPISAPSEQMVDTPASEPAQASTPTDPKGLAQL",
# CCG4_HUMAN
"MVRCDRGLQMLLTTAGAFAAFSLMAIAIGTDYWLYSSAHICNGTNLTMDDGPPPRRARGDLTHSGLWRVCCIEGIYKGHCFRINHFPEDNDYDHDSSEYLLRIVRASSVFPILSTILLLLGGLCIGAGRIYSRKNNIVLSAGILFVAAGLSNIIGIIVYISSNTGDPSDKRDEDKKNHYNYGWSFYFGALSFIVAETVGVLAVNIYIEKNKELRFKTKREFLKASSSSPYARMPSYRYRRRRSRSSSRSTEASPSRDVSPMGLKITGAIPMGELSMYTLSREPLKVTTAASYSPDQEASFLQVHDFFQQDLKEGFHVSMLNRRTTPV",
# VPS29_HUMAN
"MLVLVLGDLHIPHRCNSLPAKFKKLLVPGKIQHILCTGNLCTKESYDYLKTLAGDVHIVRGDFDENLNYPEQKVVTVGQFKIGLIHGHQVIPWGDMASLALLQRQFDVDILISGHTHKFEAFEHENKFYINPGSATGAYNALETNIIPSFVLMDIQASTVVTYVYQLIGDDVKVERIEYKKP",
# GABR1_HUMAN
"MLLLLLLAPLFLRPPGAGGAQTPNATSEGCQIIHPPWEGGIRYRGLTRDQVKAINFLPVDYEIEYVCRGEREVVGPKVRKCLANGSWTDMDTPSRCVRICSKSYLTLENGKVFLTGGDLPALDGARVDFRCDPDFHLVGSSRSICSQGQWSTPKPHCQVNRTPHSERRAVYIGALFPMSGGWPGGQACQPAVEMALEDVNSRRDILPDYELKLIHHDSKCDPGQATKYLYELLYNDPIKIILMPGCSSVSTLVAEAARMWNLIVLSYGSSSPALSNRQRFPTFFRTHPSATLHNPTRVKLFEKWGWKKIATIQQTTEVFTSTLDDLEERVKEAGIEITFRQSFFSDPAVPVKNLKRQDARIIVGLFYETEARKVFCEVYKERLFGKKYVWFLIGWYADNWFKIYDPSINCTVDEMTEAVEGHITTEIVMLNPANTRSISNMTSQEFVEKLTKRLKRHPEETGGFQEAPLAYDAIWALALALNKTSGGGGRSGVRLEDFNYNNQTITDQIYRAMNSSSFEGVSGHVVFDASGSRMAWTLIEQLQGGSYKKIGYYDSTKDDLSWSKTDKWIGGSPPADQTLVIKTFRFLSQKLFISVSVLSSLGIVLAVVCLSFNIYNSHVRYIQNSQPNLNNLTAVGCSLALAAVFPLGLDGYHIGRNQFPFVCQARLWLLGLGFSLGYGSMFTKIWWVHTVFTKKEEKKEWRKTLEPWKLYATVGLLVGMDVLTLAIWQIVDPLHRTIETFAKEEPKEDIDVSILPQLEHCSSRKMNTWLGIFYGYKGLLLLLGIFLAYETKSVSTEKINDHRAVGMAIYNVAVLCLITAPVTMILSSQQDAAFAFASLAIVFSSYITLVVLFVPKMRRLITRGEWQSEAQDTMKTGSSTNNNEEEKSRLLEKENRELEKIIAEKEERVSELRHQLQSRQQLRSRRHPPTPPEPSGGLPRGPPEPPDRLSCDGSRVHLLYK",
# GHRL_HUMAN
"MPSPGTVCSLLLLGMLWLDLAMAGSSFLSPEHQRVQQRKESKKPPAKLQPRALAGWLRPEDGGQAEGAEDELEVRFNAPFDVGIKLSGVQYQQHSQALGKFLQDILWEEAKEAPADK",
# PPR1B_HUMAN
"MDPKDRKKIQFSVPAPPSQLDPRQVEMIRRRRPTPAMLFRLSEHSSPEEEASPHQRASGEGHHLKSKRPNPCAYTPPSLKAVQRIAESHLQSISNLNENQASEEEDELGELRELGYPREEDEEEEEDDEEEEEEEDSQAEVLKVIRQSAGQKTTCGQGLEGPWERPPPLDESERDGGSEDQVEDPALSEPGEEPQRPSPSEPGT",
# VTI1B_HUMAN
"MASSAASSEHFEKLHEIFRGLHEDLQGVPERLLGTAGTEEKKKLIRDFDEKQQEANETLAEMEEELRYAPLSFRNPMMSKLRNYRKDLAKLHREVRSTPLTATPGGRGDMKYGIYAVENEHMNRLQSQRAMLLQGTESLNRATQSIERSHRIATETDQIGSEIIEELGEQRDQLERTKSRLVNTSENLSKSRKILRSMSRKVTTNKLLLSIIILLELAILGGLVYYKFFRSH",
# CCG5_HUMAN
"MSACGRKALTLLSSVFAVCGLGLLGIAVSTDYWLYLEEGVIVPQNQSTEIKMSLHSGLWRVCFLAGEERGRCFTIEYVMPMNTQLTSESTVNVLKMIRSATPFPLVSLFFMFIGFILNNIGHIRPHRTILAFVSGIFFILSGLSLVVGLVLYISSINDEMLNRTKDAETYFNYKYGWSFAFAAISFLLTESAGVMSVYLFMKRYTAEDMYRPHPGFYRPRLSNCSDYSGQFLHPDAWVRGRSPSDISSEASLQMNSNYPALLKCPDYDQMSSSPC",
# ACHA9_HUMAN
"MNWSHSCISFCWIYFAASRLRAAETADGKYAQKLFNDLFEDYSNALRPVEDTDKVLNVTLQITLSQIKDMDERNQILTAYLWIRQIWHDAYLTWDRDQYDGLDSIRIPSDLVWRPDIVLYNKADDESSEPVNTNVVLRYDGLITWDAPAITKSSCVVDVTYFPFDNQQCNLTFGSWTYNGNQVDIFNALDSGDLSDFIEDVEWEVHGMPAVKNVISYGCCSEPYPDVTFTLLLKRRSSFYIVNLLIPCVLISFLAPLSFYLPAASGEKVSLGVTILLAMTVFQLMVAEIMPASENVPLIGKYYIATMALITASTALTIMVMNIHFCGAEARPVPHWARVVILKYMSRVLFVYDVGESCLSPHHSRERDHLTKVYSKLPESNLKAARNKDLSRKKDMNKRLKNDLGCQGKNPQEAESYCAQYKVLTRNIEYIAKCLKDHKATNSKGSEWKKVAKVIDRFFMWIFFIMVFVMTILIIARAD",
# SEPT3_HUMAN
"MSKGLPETRTDAAMSELVPEPRPKPAVPMKPMSINSNLLGYIGIDTIIEQMRKKTMKTGFDFNIMVVGQSGLGKSTLVNTLFKSQVSRKASSWNREEKIPKTVEIKAIGHVIEEGGVKMKLTVIDTPGFGDQINNENCWEPIEKYINEQYEKFLKEEVNIARKKRIPDTRVHCCLYFISPTGHSLRPLDLEFMKHLSKVVNIIPVIAKADTMTLEEKSEFKQRVRKELEVNGIEFYPQKEFDEDLEDKTENDKIRQESMPFAVVGSDKEYQVNGKRVLGRKTPWGIIEVENLNHCEFALLRDFVIRTHLQDLKEVTHNIHYETYRAKRLNDNGGLPPGEGLLGTVLPPVPATPCPTAE",
# SWP70_HUMAN
"MGSLKEELLKAIWHAFTALDQDHSGKVSKSQLKVLSHNLCTVLKVPHDPVALEEHFRDDDEGPVSNQGYMPYLNRFILEKVQDNFDKIEFNRMCWTLCVKKNLTKNPLLITEEDAFKIWVIFNFLSEDKYPLIIVSEEIEYLLKKLTEAMGGGWQQEQFEHYKINFDDSKNGLSAWELIELIGNGQFSKGMDRQTVSMAINEVFNELILDVLKQGYMMKKGHRRKNWTERWFVLKPNIISYYVSEDLKDKKGDILLDENCCVESLPDKDGKKCLFLVKCFDKTFEISASDKKKKQEWIQAIHSTIHLLKLGSPPPHKEARQRRKELRKKQLAEQEELERQMKELQAANESKQQELEAVRKKLEEAASRAAEEEKKRLQTQVELQARFSTELEREKLIRQQMEEQVAQKSSELEQYLQRVRELEDMYLKLQEALEDERQARQDEETVRKLQARLLEEESSKRAELEKWHLEQQQAIQTTEAEKQELENQRVLKEQALQEAMEQLEQLELERKQALEQYEEVKKKLEMATNKTKSWKDKVAHHEGLIRLIEPGSKNPHLITNWGPAAFTEAELEEREKNWKEKKTTE",
# CNTP2_HUMAN
"MQAAPRAGCGAALLLWIVSSCLCRAWTAPSTSQKCDEPLVSGLPHVAFSSSSSISGSYSPGYAKINKRGGAGGWSPSDSDHYQWLQVDFGNRKQISAIATQGRYSSSDWVTQYRMLYSDTGRNWKPYHQDGNIWAFPGNINSDGVVRHELQHPIIARYVRIVPLDWNGEGRIGLRIEVYGCSYWADVINFDGHVVLPYRFRNKKMKTLKDVIALNFKTSESEGVILHGEGQQGDYITLELKKAKLVLSLNLGSNQLGPIYGHTSVMTGSLLDDHHWHSVVIERQGRSINLTLDRSMQHFRTNGEFDYLDLDYEITFGGIPFSGKPSSSSRKNFKGCMESINYNGVNITDLARRKKLEPSNVGNLSFSCVEPYTVPVFFNATSYLEVPGRLNQDLFSVSFQFRTWNPNGLLVFSHFADNLGNVEIDLTESKVGVHINITQTKMSQIDISSGSGLNDGQWHEVRFLAKENFAILTIDGDEASAVRTNSPLQVKTGEKYFFGGFLNQMNNSSHSVLQPSFQGCMQLIQVDDQLVNLYEVAQRKPGSFANVSIDMCAIIDRCVPNHCEHGGKCSQTWDSFKCTCDETGYSGATCHNSIYEPSCEAYKHLGQTSNYYWIDPDGSGPLGPLKVYCNMTEDKVWTIVSHDLQMQTPVVGYNPEKYSVTQLVYSASMDQISAITDSAEYCEQYVSYFCKMSRLLNTPDGSPYTWWVGKANEKHYYWGGSGPGIQKCACGIERNCTDPKYYCNCDADYKQWRKDAGFLSYKDHLPVSQVVVGDTDRQGSEAKLSVGPLRCQGDRNYWNAASFPNPSSYLHFSTFQGETSADISFYFKTLTPWGVFLENMGKEDFIKLELKSATEVSFSFDVGNGPVEIVVRSPTPLNDDQWHRVTAERNVKQASLQVDRLPQQIRKAPTEGHTRLELYSQLFVGGAGGQQGFLGCIRSLRMNGVTLDLEERAKVTSGFISGCSGHCTSYGTNCENGGKCLERYHGYSCDCSNTAYDGTFCNKDVGAFFEEGMWLRYNFQAPATNARDSSSRVDNAPDQQNSHPDLAQEEIRFSFSTTKAPCILLYISSFTTDFLAVLVKPTGSLQIRYNLGGTREPYNIDVDHRNMANGQPHSVNITRHEKTIFLKLDHYPSVSYHLPSSSDTLFNSPKSLFLGKVIETGKIDQEIHKYNTPGFTGCLSRVQFNQIAPLKAALRQTNASAHVHIQGELVESNCGASPLTLSPMSSATDPWHLDHLDSASADFPYNPGQGQAIRNGVNRNSAIIGGVIAVVIFTILCTLVFLIRYMFRHKGTYHTNEAKGAESAESADAAIMNNDPNFTETIDESKKEWLI",
# SEPT9_HUMAN
"MKKSYSGGTRTSSGRLRRLGDSSGPALKRSFEVEEVETPNSTPPRRVQTPLLRATVASSTQKFQDLGVKNSEPSARHVDSLSQRSPKASLRRVELSGPKAAEPVSRRTELSIDISSKQVENAGAIGPSRFGLKRAEVLGHKTPEPAPRRTEITIVKPQESAHRRMEPPASKVPEVPTAPATDAAPKRVEIQMPKPAEAPTAPSPAQTLENSEPAPVSQLQSRLEPKPQPPVAEATPRSQEATEAAPSCVGDMADTPRDAGLKQAPASRNEKAPVDFGYVGIDSILEQMRRKAMKQGFEFNIMVVGQSGLGKSTLINTLFKSKISRKSVQPTSEERIPKTIEIKSITHDIEEKGVRMKLTVIDTPGFGDHINNENCWQPIMKFINDQYEKYLQEEVNINRKKRIPDTRVHCCLYFIPATGHSLRPLDIEFMKRLSKVVNIVPVIAKADTLTLEERVHFKQRITADLLSNGIDVYPQKEFDEDSEDRLVNEKFREMIPFAVVGSDHEYQVNGKRILGRKTKWGTIEVENTTHCEFAYLRDLLIRTHMQNIKDITSSIHFEAYRVKRLNEGSSAMANGMEEKEPEAPEM",
# NUFP1_HUMAN
"MAEPTSDFETPIGWHASPELTPTLGPLSDTAPPRDSWMFWAMLPPPPPPLTSSLPAAGSKPSSESQPPMEAQSLPGAPPPFDAQILPGAQPPFDAQSPLDSQPQPSGQPWNFHASTSWYWRQSSDRFPRHQKSFNPAVKNSYYPRKYDAKFTDFSLPPSRKQKKKKRKEPVFHFFCDTCDRGFKNQEKYDKHMSEHTKCPELDCSFTAHEKIVQFHWRNMHAPGMKKIKLDTPEEIARWREERRKNYPTLANIERKKKLKLEKEKRGAVLTTTQYGKMKGMSRHSQMAKIRSPGKNHKWKNDNSRQRAVTGSGSHLCDLKLEGPPEANADPLGVLINSDSESDKEEKPQHSVIPKEVTPALCSLMSSYGSLSGSESEPEETPIKTEADVLAENQVLDSSAPKSPSQDVKATVRNFSEAKSENRKKSFEKTNPKRKKDYHNYQTLFEPRTHHPYLLEMLLAPDIRHERNVILQCVRYIIKKDFFGLDTNSAKSKDV",
# VATH_HUMAN
"MTKMDIRGAVDAAVPTNIIAAKAAEVRANKVNWQSYLQGQMISAEDCEFIQRFEMKRSPEEKQEMLQTEGSQCAKTFINLMTHICKEQTVQYILTMVDDMLQENHQRVSIFFDYARCSKNTAWPYFLPMLNRQDPFTVHMAARIIAKLAAWGKELMEGSDLNYYFNWIKTQLSSQKLRGSGVAVETGTVSSSDSSQYVQCVAGCLQLMLRVNEYRFAWVEADGVNCIMGVLSNKCGFQLQYQMIFSIWLLAFSPQMCEHLRRYNIIPVLSDILQESVKEKVTRIILAAFRNFLEKSTERETRQEYALAMIQCKVLKQLENLEQQKYDDEDISEDIKFLLEKLGESVQDLSSFDEYSSELKSGRLEWSPVHKSEKFWRENAVRLNEKNYELLKILTKLLEVSDDPQVLAVAAHDVGEYVRHYPRGKRVIEQLGGKQLVMNHMHHEDQQVRYNALLAVQKLMVHNWEYLGKQLQSEQPQTAAARS",
# PRAF1_HUMAN
"MAAQKDQQKDAEAEGLSGTTLLPKLIPSGAGREWLERRRATIRPWSTFVDQQRFSRPRNLGELCQRLVRNVEYYQSNYVFVFLGLILYCVVTSPMLLVALAVFFGACYILYLRTLESKLVLFGREVSPAHQYALAGGISFPFFWLAGAGSAVFWVLGATLVVIGSHAAFHQIEAVDGEELQMEPV",
# TAGL3_HUMAN
"MANRGPSYGLSREVQEKIEQKYDADLENKLVDWIILQCAEDIEHPPPGRAHFQKWLMDGTVLCKLINSLYPPGQEPIPKISESKMAFKQMEQISQFLKAAETYGVRTTDIFQTVDLWEGKDMAAVQRTLMALGSVAVTKDDGCYRGEPSWFHRKAQQNRRGFSEEQLRQGQNVIGLQMGSNKGASQAGMTGYGMPRQIM",
# SCNBA_HUMAN
"MDDRCYPVIFPDERNFRPFTSDSLAAIEKRIAIQKEKKKSKDQTGEVPQPRPQLDLKASRKLPKLYGDIPRELIGKPLEDLDPFYRNHKTFMVLNRKRTIYRFSAKHALFIFGPFNSIRSLAIRVSVHSLFSMFIIGTVIINCVFMATGPAKNSNSNNTDIAECVFTGIYIFEALIKILARGFILDEFSFLRDPWNWLDSIVIGIAIVSYIPGITIKLLPLRTFRVFRALKAISVVSRLKVIVGALLRSVKKLVNVIILTFFCLSIFALVGQQLFMGSLNLKCISRDCKNISNPEAYDHCFEKKENSPEFKMCGIWMGNSACSIQYECKHTKINPDYNYTNFDNFGWSFLAMFRLMTQDSWEKLYQQTLRTTGLYSVFFFIVVIFLGSFYLINLTLAVVTMAYEEQNKNVAAEIEAKEKMFQEAQQLLKEEKEALVAMGIDRSSLTSLETSYFTPKKRKLFGNKKRKSFFLRESGKDQPPGSDSDEDCQKKPQLLEQTKRLSQNLSLDHFDEHGDPLQRQRALSAVSILTITMKEQEKSQEPCLPCGENLASKYLVWNCCPQWLCVKKVLRTVMTDPFTELAITICIIINTVFLAMEHHKMEASFEKMLNIGNLVFTSIFIAEMCLKIIALDPYHYFRRGWNIFDSIVALLSFADVMNCVLQKRSWPFLRSFRVLRVFKLAKSWPTLNTLIKIIGNSVGALGSLTVVLVIVIFIFSVVGMQLFGRSFNSQKSPKLCNPTGPTVSCLRHWHMGDFWHSFLVVFRILCGEWIENMWECMQEANASSSLCVIVFILITVIGKLVVLNLFIALLLNSFSNEERNGNLEGEARKTKVQLALDRFRRAFCFVRHTLEHFCHKWCRKQNLPQQKEVAGGCAAQSKDIIPLVMEMKRGSETQEELGILTSVPKTLGVRHDWTWLAPLAEEEDDVEFSGEDNAQRITQPEPEQQAYELHQENKKPTSQRVQSVEIDMFSEDEPHLTIQDPRKKSDVTSILSECSTIDLQDGFGWLPEMVPKKQPERCLPKGFGCCFPCCSVDKRKPPWVIWWNLRKTCYQIVKHSWFESFIIFVILLSSGALIFEDVHLENQPKIQELLNCTDIIFTHIFILEMVLKWVAFGFGKYFTSAWCCLDFIIVIVSVTTLINLMELKSFRTLRALRPLRALSQFEGMKVVVNALIGAIPAILNVLLVCLIFWLVFCILGVYFFSGKFGKCINGTDSVINYTIITNKSQCESGNFSWINQKVNFDNVGNAYLALLQVATFKGWMDIIYAAVDSTEKEQQPEFESNSLGYIYFVVFIIFGSFFTLNLFIGVIIDNFNQQQKKLGGQDIFMTEEQKKYYNAMKKLGSKKPQKPIPRPLNKCQGLVFDIVTSQIFDIIIISLIILNMISMMAESYNQPKAMKSILDHLNWVFVVIFTLECLIKIFALRQYYFTNGWNLFDCVVVLLSIVSTMISTLENQEHIPFPPTLFRIVRLARIGRILRLVRAARGIRTLLFALMMSLPSLFNIGLLLFLIMFIYAILGMNWFSKVNPESGIDDIFNFKTFASSMLCLFQISTSAGWDSLLSPMLRSKESCNSSSENCHLPGIATSYFVSYIIISFLIVVNMYIAVILENFNTATEESEDPLGEDDFDIFYEVWEKFDPEATQFIKYSALSDFADALPEPLRVAKPNKYQFLVMDLPMVSEDRLHCMDILFAFTARVLGGSDGLDSMKAMMEEKFMEANPLKKLYEPIVTTTKRKEEERGAAIIQKAFRKYMMKVTKGDQGDQNDLENGPHSPLQTLCNGDLSSFGVAKGKVHCD",
# NCKX2_HUMAN
"MDLQQSTTITSLEKWCLDESLSGCRRHYSVKKKLKLIRVLGLFMGLVAISTVSFSISAFSETDTQSTGEASVVSGPRVAQGYHQRTLLDLNDKILDYTPQPPLSKEGESENSTDHAQGDYPKDIFSLEERRKGAIILHVIGMIYMFIALAIVCDEFFVPSLTVITEKLGISDDVAGATFMAAGGSAPELFTSLIGVFIAHSNVGIGTIVGSAVFNILFVIGMCALFSREILNLTWWPLFRDVSFYIVDLIMLIIFFLDNVIMWWESLLLLTAYFCYVVFMKFNVQVEKWVKQMINRNKVVKVTAPEAQAKPSAARDKDEPTLPAKPRLQRGGSSASLHNSLMRNSIFQLMIHTLDPLAEELGSYGKLKYYDTMTEEGRFREKASILHKIAKKKCHVDENERQNGAANHVEKIELPNSTSTDVEMTPSSDASEPVQNGNLSHNIEGAEAQTADEEEDQPLSLAWPSETRKQVTFLIVFPIVFPLWITLPDVRKPSSRKFFPITFFGSITWIAVFSYLMVWWAHQVGETIGISEEIMGLTILAAGTSIPDLITSVIVARKGLGDMAVSSSVGSNIFDITVGLPLPWLLYTVIHRFQPVAVSSNGLFCAIVLLFIMLLFVILSIALCKWRMNKILGFIMFGLYFVFLVVSVLLEDRILTCPVSI",
# ZDHC2_HUMAN
"MAPSGPGSSARRRCRRVLYWIPVVFITLLLGWSYYAYAIQLCIVSMENTGEQVVCLMAYHLLFAMFVWSYWKTIFTLPMNPSKEFHLSYAEKDLLEREPRGEAHQEVLRRAAKDLPIYTRTMSGAIRYCDRCQLIKPDRCHHCSVCDKCILKMDHHCPWVNNCVGFSNYKFFLLFLAYSLLYCLFIAATDLQYFIKFWTNGLPDTQAKFHIMFLFFAAAMFSVSLSSLFGYHCWLVSKNKSTLEAFRSPVFRHGTDKNGFSLGFSKNMRQVFGDEKKYWLLPIFSSLGDGCSFPTCLVNQDPEQASTPAGLNSTAKNLENHQFPAKPLRESQSHLLTDSQSWTESSINPGKCKAGMSNPALTMENET",
# PLXA1_HUMAN
"MPLPPRSLQVLLLLLLLLLLLPGMWAEAGLPRAGGGSQPPFRTFSASDWGLTHLVVHEQTGEVYVGAVNRIYKLSGNLTLLRAHVTGPVEDNEKCYPPPSVQSCPHGLGSTDNVNKLLLLDYAANRLLACGSASQGICQFLRLDDLFKLGEPHHRKEHYLSSVQEAGSMAGVLIAGPPGQGQAKLFVGTPIDGKSEYFPTLSSRRLMANEEDADMFGFVYQDEFVSSQLKIPSDTLSKFPAFDIYYVYSFRSEQFVYYLTLQLDTQLTSPDAAGEHFFTSKIVRLCVDDPKFYSYVEFPIGCEQAGVEYRLVQDAYLSRPGRALAHQLGLAEDEDVLFTVFAQGQKNRVKPPKESALCLFTLRAIKEKIKERIQSCYRGEGKLSLPWLLNKELGCINSPLQIDDDFCGQDFNQPLGGTVTIEGTPLFVDKDDGLTAVAAYDYRGRTVVFAGTRSGRIRKILVDLSNPGGRPALAYESVVAQEGSPILRDLVLSPNHQYLYAMTEKQVTRVPVESCVQYTSCELCLGSRDPHCGWCVLHSICSRRDACERADEPQRFAADLLQCVQLTVQPRNVSVTMSQVPLVLQAWNVPDLSAGVNCSFEDFTESESVLEDGRIHCRSPSAREVAPITRGQGDQRVVKLYLKSKETGKKFASVDFVFYNCSVHQSCLSCVNGSFPCHWCKYRHVCTHNVADCAFLEGRVNVSEDCPQILPSTQIYVPVGVVKPITLAARNLPQPQSGQRGYECLFHIPGSPARVTALRFNSSSLQCQNSSYSYEGNDVSDLPVNLSVVWNGNFVIDNPQNIQAHLYKCPALRESCGLCLKADPRFECGWCVAERRCSLRHHCAADTPASWMHARHGSSRCTDPKILKLSPETGPRQGGTRLTITGENLGLRFEDVRLGVRVGKVLCSPVESEYISAEQIVCEIGDASSVRAHDALVEVCVRDCSPHYRALSPKRFTFVTPTFYRVSPSRGPLSGGTWIGIEGSHLNAGSDVAVSVGGRPCSFSWRNSREIRCLTPPGQSPGSAPIIININRAQLTNPEVKYNYTEDPTILRIDPEWSINSGGTLLTVTGTNLATVREPRIRAKYGGIERENGCLVYNDTTMVCRAPSVANPVRSPPELGERPDELGFVMDNVRSLLVLNSTSFLYYPDPVLEPLSPTGLLELKPSSPLILKGRNLLPPAPGNSRLNYTVLIGSTPCTLTVSETQLLCEAPNLTGQHKVTVRAGGFEFSPGTLQVYSDSLLTLPAIVGIGGGGGLLLLVIVAVLIAYKRKSRDADRTLKRLQLQMDNLESRVALECKEAFAELQTDIHELTNDLDGAGIPFLDYRTYAMRVLFPGIEDHPVLKEMEVQANVEKSLTLFGQLLTKKHFLLTFIRTLEAQRSFSMRDRGNVASLIMTALQGEMEYATGVLKQLLSDLIEKNLESKNHPKLLLRRTESVAEKMLTNWFTFLLYKFLKECAGEPLFMLYCAIKQQMEKGPIDAITGEARYSLSEDKLIRQQIDYKTLTLNCVNPENENAPEVPVKGLDCDTVTQAKEKLLDAAYKGVPYSQRPKAADMDLEWRQGRMARIILQDEDVTTKIDNDWKRLNTLAHYQVTDGSSVALVPKQTSAYNISNSSTFTKSLSRYESMLRTASSPDSLRSRTPMITPDLESGTKLWHLVKNHDHLDQREGDRGSKMVSEIYLTRLLATKGTLQKFVDDLFETIFSTAHRGSALPLAIKYMFDFLDEQADKHQIHDADVRHTWKSNCLPLRFWVNVIKNPQFVFDIHKNSITDACLSVVAQTFMDSCSTSEHKLGKDSPSNKLLYAKDIPNYKSWVERYYADIAKMPAISDQDMSAYLAEQSRLHLSQFNSMSALHEIYSYITKYKDEILAALEKDEQARRQRLRSKLEQVVDTMALSS",
# RABX5_HUMAN
"MSLKSERRGIHVDQSDLLCKKGCGYYGNPAWQGFCSKCWREEYHKARQKQIQEDWELAERLQREEEEAFASSQSSQGAQSLTFSKFEEKKTNEKTRKVTTVKKFFSASSRVGSKKEIQEAKAPSPSINRQTSIETDRVSKEFIEFLKTFHKTGQEIYKQTKLFLEGMHYKRDLSIEEQSECAQDFYHNVAERMQTRGKVPPERVEKIMDQIEKYIMTRLYKYVFCPETTDDEKKDLAIQKRIRALRWVTPQMLCVPVNEDIPEVSDMVVKAITDIIEMDSKRVPRDKLACITKCSKHIFNAIKITKNEPASADDFLPTLIYIVLKGNPPRLQSNIQYITRFCNPSRLMTGEDGYYFTNLCCAVAFIEKLDAQSLNLSQEDFDRYMSGQTSPRKQEAESWSPDACLGVKQMYKNLDLLSQLNERQERIMNEAKKLEKDLIDWTDGIAREVQDIVEKYPLEIKPPNQPLAAIDSENVENDKLPPPLQPQVYAG",
# RIMS3_HUMAN
"MFNGEPGPASSGASRNVVRSSSISGEICGSQQAGGGAGTTTAKKRRSSLGAKMVAIVGLTQWSKSTLQLPQPEGATKKLRSNIRRSTETGIAVEMRSRVTRQGSRESTDGSTNSNSSDGTFIFPTTRLGAESQFSDFLDGLGPAQIVGRQTLATPPMGDVHIAIMDRSGQLEVEVIEARGLTPKPGSKSLPATYIKVYLLENGACLAKKKTKMTKKTCDPLYQQALLFDEGPQGKVLQVIVWGDYGRMDHKCFMGMAQIMLDELDLSAAVTGWYKLFPTSSVADSTLGSLTRRLSQSSLESATSPSCS",
# LAMP5_HUMAN
"MDLQGRGVPSIDRLRVLLMLFHTMAQIMAEQEVENLSGLSTNPEKDIFVVRENGTTCLMAEFAAKFIVPYDVWASNYVDLITEQADIALTRGAEVKGRCGHSQSELQVFWVDRAYALKMLFVKESHNMSKGPEATWRLSKVQFVYDSSEKTHFKDAVSAGKHTANSHHLSALVTPAGKSYECQAQQTISLASSDPQKTVTMILSAVHIQPFDIISDFVFSEEHKCPVDEREQLEETLPLILGLILGLVIMVTLAIYHVHHKMTANQVQIPRDRSQYKHMG",
# DBNL_HUMAN
"MAANLSRNGPALQEAYVRVVTEKSPTDWALFTYEGNSNDIRVAGTGEGGLEEMVEELNSGKVMYAFCRVKDPNSGLPKFVLINWTGEGVNDVRKGACASHVSTMASFLKGAHVTINARAEEDVEPECIMEKVAKASGANYSFHKESGRFQDVGPQAPVGSVYQKTNAVSEIKRVGKDSFWAKAEKEEENRRLEEKRRAEEAQRQLEQERRERELREAARREQRYQEQGGEASPQRTWEQQQEVVSRNRNEQESAVHPREIFKQKERAMSTTSISSPQPGKLRSPFLQKQLTQPETHFGREPAAAISRPRADLPAEEPAPSTPPCLVQAEEEAVYEEPPEQETFYEQPPLVQQQGAGSEHIDHHIQGQGLSGQGLCARALYDYQAADDTEISFDPENLITGIEVIDEGWWRGYGPDGHFGMFPANYVELIE",
# PURG_HUMAN
"MERARRRGGGGGRGRGGKNVGGSGLSKSRLYPQAQHSHYPHYAASATPNQAGGAAEIQELASKRVDIQKKRFYLDVKQSSRGRFLKIAEVWIGRGRQDNIRKSKLTLSLSVAAELKDCLGDFIEHYAHLGLKGHRQEHGHSKEQGSRRRQKHSAPSPPVSVGSEEHPHSVLKTDYIERDNRKYYLDLKENQRGRFLRIRQTMMRGTGMIGYFGHSLGQEQTIVLPAQGMIEFRDALVQLIEDYGEGDIEERRGGDDDPLELPEGTSFRVDNKRFYFDVGSNKYGIFLKVSEVRPPYRNTITVPFKAWTRFGENFIKYEEEMRKICNSHKEKRMDGRKASGEEQECLD",
# ANC2_HUMAN
"MAAAVVVAEGDSDSRPGQELLVAWNTVSTGLVPPAALGLVSSRTSGAVPPKEEELRAAVEVLRGHGLHSVLEEWFVEVLQNDLQANISPEFWNAISQCENSADEPQCLLLLLDAFGLLESRLDPYLRSLELLEKWTRLGLLMGTGAQGLREEVHTMLRGVLFFSTPRTFQEMIQRLYGCFLRVYMQSKRKGEGGTDPELEGELDSRYARRRYYRLLQSPLCAGCSSDKQQCWCRQALEQFHQLSQVLHRLSLLERVSAEAVTTTLHQVTRERMEDRCRGEYERSFLREFHKWIERVVGWLGKVFLQDGPARPASPEAGNTLRRWRCHVQRFFYRIYASLRIEELFSIVRDFPDSRPAIEDLKYCLERTDQRQQLLVSLKAALETRLLHPGVNTCDIITLYISAIKALRVLDPSMVILEVACEPIRRYLRTREDTVRQIVAGLTGDSDGTGDLAVELSKTDPASLETGQDSEDDSGEPEDWVPDPVDADPGKSSSKRRSSDIISLLVSIYGSKDLFINEYRSLLADRLLHQFSFSPEREIRNVELLKLRFGEAPMHFCEVMLKDMADSRRINANIREEDEKRPAEEQPPFGVYAVILSSEFWPPFKDEKLEVPEDIRAALEAYCKKYEQLKAMRTLSWKHTLGLVTMDVELADRTLSVAVTPVQAVILLYFQDQASWTLEELSKAVKMPVALLRRRMSVWLQQGVLREEPPGTFSVIEEERPQDRDNMVLIDSDDESDSGMASQADQKEEELLLFWTYIQAMLTNLESLSLDRIYNMLRMFVVTGPALAEIDLQELQGYLQKKVRDQQLVYSAGVYRLPKNCS",
# KCND3_HUMAN
"MAAGVAAWLPFARAAAIGWMPVANCPMPLAPADKNKRQDELIVLNVSGRRFQTWRTTLERYPDTLLGSTEKEFFFNEDTKEYFFDRDPEVFRCVLNFYRTGKLHYPRYECISAYDDELAFYGILPEIIGDCCYEEYKDRKRENAERLMDDNDSENNQESMPSLSFRQTMWRAFENPHTSTLALVFYYVTGFFIAVSVITNVVETVPCGTVPGSKELPCGERYSVAFFCLDTACVMIFTVEYLLRLFAAPSRYRFIRSVMSIIDVVAIMPYYIGLVMTNNEDVSGAFVTLRVFRVFRIFKFSRHSQGLRILGYTLKSCASELGFLLFSLTMAIIIFATVMFYAEKGSSASKFTSIPASFWYTIVTMTTLGYGDMVPKTIAGKIFGSICSLSGVLVIALPVPVIVSNFSRIYHQNQRADKRRAQKKARLARIRVAKTGSSNAYLHSKRNGLLNEALELTGTPEEEHMGKTTSLIESQHHHLLHCLEKTTGLSYLVDDPLLSVRTSTIKNHEFIDEQMFEQNCMESSMQNYPSTRSPSLSSHPGLTTTCCSRRSKKTTHLPNSNLPATRLRSMQELSTIHIQGSEQPSLTTSRSSLNLKADDGLRPNCKTSQITTAIISIPTPPALTPEGESRPPPASPGPNTNIPSIASNVVKVSAL",
# FBX2_HUMAN
"MDGDGDPESVGQPEEASPEEQPEEASAEEERPEDQQEEEAAAAAAYLDELPEPLLLRVLAALPAAELVQACRLVCLRWKELVDGAPLWLLKCQQEGLVPEGGVEEERDHWQQFYFLSKRRRNLLRNPCGEEDLEGWCDVEHGGDGWRVEELPGDSGVEFTHDESVKKYFASSFEWCRKAQVIDLQAEGYWEELLDTTQPAIVVKDWYSGRSDAGCLYELTVKLLSEHENVLAEFSSGQVAVPQDSDGGGWMEISHTFTDYGPGVRFVRFEHGGQDSVYWKGWFGARVTNSSVWVEP",
# KS6A6_HUMAN
"MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSCHDEGVVKEIPITHHVKEGYEKADPAQFELLKVLGQGSFGKVFLVRKKTGPDAGQLYAMKVLKKASLKVRDRVRTKMERDILVEVNHPFIVKLHYAFQTEGKLYLILDFLRGGDVFTRLSKEVLFTEEDVKFYLAELALALDHLHQLGIVYRDLKPENILLDEIGHIKLTDFGLSKESVDQEKKAYSFCGTVEYMAPEVVNRRGHSQSADWWSYGVLMFEMLTGTLPFQGKDRNETMNMILKAKLGMPQFLSAEAQSLLRMLFKRNPANRLGSEGVEEIKRHLFFANIDWDKLYKREVQPPFKPASGKPDDTFCFDPEFTAKTPKDSPGLPASANAHQLFKGFSFVATSIAEEYKITPITSANVLPIVQINGNAAQFGEVYELKEDIGVGSYSVCKRCIHATTNMEFAVKIIDKSKRDPSEEIEILMRYGQHPNIITLKDVFDDGRYVYLVTDLMKGGELLDRILKQKCFSEREASDILYVISKTVDYLHCQGVVHRDLKPSNILYMDESASADSIRICDFGFAKQLRGENGLLLTPCYTANFVAPEVLMQQGYDAACDIWSLGVLFYTMLAGYTPFANGPNDTPEEILLRIGNGKFSLSGGNWDNISDGAKDLLSHMLHMDPHQRYTAEQILKHSWITHRDQLPNDQPKRNDVSHVVKGAMVATYSALTHKTFQPVLEPVAASSLAQRRSMKKRTSTGL",
# TNIK_HUMAN
"MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVKTGQLAAIKVMDVTGDEEEEIKQEINMLKKYSHHRNIATYYGAFIKKNPPGMDDQLWLVMEFCGAGSVTDLIKNTKGNTLKEEWIAYICREILRGLSHLHQHKVIHRDIKGQNVLLTENAEVKLVDFGVSAQLDRTVGRRNTFIGTPYWMAPEVIACDENPDATYDFKSDLWSLGITAIEMAEGAPPLCDMHPMRALFLIPRNPAPRLKSKKWSKKFQSFIESCLVKNHSQRPATEQLMKHPFIRDQPNERQVRIQLKDHIDRTKKKRGEKDETEYEYSGSEEEEEENDSGEPSSILNLPGESTLRRDFLRLQLANKERSEALRRQQLEQQQRENEEHKRQLLAERQKRIEEQKEQRRRLEEQQRREKELRKQQEREQRRHYEEQMRREEERRRAEHEQEYIRRQLEEEQRQLEILQQQLLHEQALLLEYKRKQLEEQRQAERLQRQLKQERDYLVSLQHQRQEQRPVEKKPLYHYKEGMSPSEKPAWAKEVEERSRLNRQSSPAMPHKVANRISDPNLPPRSESFSISGVQPARTPPMLRPVDPQIPHLVAVKSQGPALTASQSVHEQPTKGLSGFQEALNVTSHRVEMPRQNSDPTSENPPLPTRIEKFDRSSWLRQEEDIPPKVPQRTTSISPALARKNSPGNGSALGPRLGSQPIRASNPDLRRTEPILESPLQRTSSGSSSSSSTPSSQPSSQGGSQPGSQAGSSERTRVRANSKSEGSPVLPHEPAKVKPEESRDITRPSRPASYKKAIDEDLTALAKELRELRIEETNRPMKKVTDYSSSSEESESSEEEEEDGESETHDGTVAVSDIPRLIPTGAPGSNEQYNVGMVGTHGLETSHADSFSGSISREGTLMIRETSGEKKRSGHSDSNGFAGHINLPDLVQQSHSPAGTPTEGLGRVSTHSQEMDSGTEYGMGSSTKASFTPFVDPRVYQTSPTDEDEEDEESSAAALFTSELLRQEQAKLNEARKISVVNVNPTNIRPHSDTPEIRKYKKRFNSEILCAALWGVNLLVGTENGLMLLDRSGQGKVYNLINRRRFQQMDVLEGLNVLVTISGKKNKLRVYYLSWLRNRILHNDPEVEKKQGWITVGDLEGCIHYKVVKYERIKFLVIALKNAVEIYAWAPKPYHKFMAFKSFADLQHKPLLVDLTVEEGQRLKVIFGSHTGFHVIDVDSGNSYDIYIPSHIQGNITPHAIVILPKTDGMEMLVCYEDEGVYVNTYGRITKDVVLQWGEMPTSVAYIHSNQIMGWGEKAIEIRSVETGHLDGVFMHKRAQRLKFLCERNDKVFFASVRSGGSSQVFFMTLNRNSMMNW",
# DP13A_HUMAN
"MPGIDKLPIEETLEDSPQTRSLLGVFEEDATAISNYMNQLYQAMHRIYDAQNELSAATHLTSKLLKEYEKQRFPLGGDDEVMSSTLQQFSKVIDELSSCHAVLSTQLADAMMFPITQFKERDLKEILTLKEVFQIASNDHDAAINRYSRLSKKRENDKVKYEVTEDVYTSRKKQHQTMMHYFCALNTLQYKKKIALLEPLLGYMQAQISFFKMGSENLNEQLEEFLANIGTSVQNVRREMDSDIETMQQTIEDLEVASDPLYVPDPDPTKFPVNRNLTRKAGYLNARNKTGLVSSTWDRQFYFTQGGNLMSQARGDVAGGLAMDIDNCSVMAVDCEDRRYCFQITSFDGKKSSILQAESKKDHEEWICTINNISKQIYLSENPEETAARVNQSALEAVTPSPSFQQRHESLRPAAGQSRPPTARTSSSGSLGSESTNLAALSLDSLVAPDTPIQFDIISPVCEDQPGQAKAFGQGGRRTNPFGESGGSTKSETEDSILHQLFIVRFLGSMEVKSDDHPDVVYETMRQILAARAIHNIFRMTESHLLVTCDCLKLIDPQTQVTRLTFPLPCVVLYATHQENKRLFGFVLRTSSGRSESNLSSVCYIFESNNEGEKICDSVGLAKQIALHAELDRRASEKQKEIERVKEKQQKELNKQKQIEKDLEEQSRLIAASSRPNQASSEGQFVVLSSSQSEESDLGEGGKKRESEA",
# ADA28_HUMAN
"MLQGLLPVSLLLSVAVSAIKELPGVKKYEVVYPIRLHPLHKREAKEPEQQEQFETELKYKMTINGKIAVLYLKKNKNLLAPGYTETYYNSTGKEITTSPQIMDDCYYQGHILNEKVSDASISTCRGLRGYFSQGDQRYFIEPLSPIHRDGQEHALFKYNPDEKNYDSTCGMDGVLWAHDLQQNIALPATKLVKLKDRKVQEHEKYIEYYLVLDNGEFKRYNENQDEIRKRVFEMANYVNMLYKKLNTHVALVGMEIWTDKDKIKITPNASFTLENFSKWRGSVLSRRKRHDIAQLITATELAGTTVGLAFMSTMCSPYSVGVVQDHSDNLLRVAGTMAHEMGHNFGMFHDDYSCKCPSTICVMDKALSFYIPTDFSSCSRLSYDKFFEDKLSNCLFNAPLPTDIISTPICGNQLVEMGEDCDCGTSEECTNICCDAKTCKIKATFQCALGECCEKCQFKKAGMVCRPAKDECDLPEMCNGKSGNCPDDRFQVNGFPCHHGKGHCLMGTCPTLQEQCTELWGPGTEVADKSCYNRNEGGSKYGYCRRVDDTLIPCKANDTMCGKLFCQGGSDNLPWKGRIVTFLTCKTFDPEDTSQEIGMVANGTKCGDNKVCINAECVDIEKAYKSTNCSSKCKGHAVCDHELQCQCEEGWIPPDCDDSSVVFHFSIVVGVLFPMAVIFVVVAMVIRHQSSREKQKKDQRPLSTTGTRPHKQKRKPQMVKAVQPQEMSQMKPHVYDLPVEGNEPPASFHKDTNALPPTVFKDNPVSTPKDSNPKA",
# AGO2_HUMAN
"MYSGAGPALAPPAPPPPIQGYAFKPPPRPDFGTSGRTIKLQANFFEMDIPKIDIYHYELDIKPEKCPRRVNREIVEHMVQHFKTQIFGDRKPVFDGRKNLYTAMPLPIGRDKVELEVTLPGEGKDRIFKVSIKWVSCVSLQALHDALSGRLPSVPFETIQALDVVMRHLPSMRYTPVGRSFFTASEGCSNPLGGGREVWFGFHQSVRPSLWKMMLNIDVSATAFYKAQPVIEFVCEVLDFKSIEEQQKPLTDSQRVKFTKEIKGLKVEITHCGQMKRKYRVCNVTRRPASHQTFPLQQESGQTVECTVAQYFKDRHKLVLRYPHLPCLQVGQEQKHTYLPLEVCNIVAGQRCIKKLTDNQTSTMIRATARSAPDRQEEISKLMRSASFNTDPYVREFGIMVKDEMTDVTGRVLQPPSILYGGRNKAIATPVQGVWDMRNKQFHTGIEIKVWAIACFAPQRQCTEVHLKSFTEQLRKISRDAGMPIQGQPCFCKYAQGADSVEPMFRHLKNTYAGLQLVVVILPGKTPVYAEVKRVGDTVLGMATQCVQMKNVQRTTPQTLSNLCLKINVKLGGVNNILLPQGRPPVFQQPVIFLGADVTHPPAGDGKKPSIAAVVGSMDAHPNRYCATVRVQQHRQEIIQDLAAMVRELLIQFYKSTRFKPTRIIFYRDGVSEGQFQQVLHHELLAIREACIKLEKDYQPGITFIVVQKRHHTRLFCTDKNERVGKSGNIPAGTTVDTKITHPTEFDFYLCSHAGIQGTSRPSHYHVLWDDNRFSSDELQILTYQLCHTYVRCTRSVSIPAPAYYAHLVAFRARYHLVDKEHDSAEGSHTSGQSNGRDHQALAKAVQVHQDTLRTMYFA",
# TEN1_HUMAN
"MEQTDCKPYQPLPKVKHEMDLAYTSSSDESEDGRKPRQSYNSRETLHEYNQELRMNYNSQSRKRKEVEKSTQEMEFCETSHTLCSGYQTDMHSVSRHGYQLEMGSDVDTETEGAASPDHALRMWIRGMKSEHSSCLSSRANSALSLTDTDHERKSDGENGFKFSPVCCDMEAQAGSTQDVQSSPHNQFTFRPLPPPPPPPHACTCARKPPPAADSLQRRSMTTRSQPSPAAPAPPTSTQDSVHLHNSWVLNSNIPLETRHFLFKHGSGSSAIFSAASQNYPLTSNTVYSPPPRPLPRSTFSRPAFTFNKPYRCCNWKCTALSATAITVTLALLLAYVIAVHLFGLTWQLQPVEGELYANGVSKGNRGTESMDTTYSPIGGKVSDKSEKKVFQKGRAIDTGEVDIGAQVMQTIPPGLFWRFQITIHHPIYLKFNISLAKDSLLGIYGRRNIPPTHTQFDFVKLMDGKQLVKQDSKGSDDTQHSPRNLILTSLQETGFIEYMDQGPWYLAFYNDGKKMEQVFVLTTAIEIMDDCSTNCNGNGECISGHCHCFPGFLGPDCARDSCPVLCGGNGEYEKGHCVCRHGWKGPECDVPEEQCIDPTCFGHGTCIMGVCICVPGYKGEICEEEDCLDPMCSNHGICVKGECHCSTGWGGVNCETPLPVCQEQCSGHGTFLLDAGVCSCDPKWTGSDCSTELCTMECGSHGVCSRGICQCEEGWVGPTCEERSCHSHCTEHGQCKDGKCECSPGWEGDHCTIAHYLDAVRDGCPGLCFGNGRCTLDQNGWHCVCQVGWSGTGCNVVMEMLCGDNLDNDGDGLTDCVDPDCCQQSNCYISPLCQGSPDPLDLIQQSQTLFSQHTSRLFYDRIKFLIGKDSTHVIPPEVSFDSRRACVIRGQVVAIDGTPLVGVNVSFLHHSDYGFTISRQDGSFDLVAIGGISVILIFDRSPFLPEKRTLWLPWNQFIVVEKVTMQRVVSDPPSCDISNFISPNPIVLPSPLTSFGGSCPERGTIVPELQVVQEEIPIPSSFVRLSYLSSRTPGYKTLLRILLTHSTIPVGMIKVHLTVAVEGRLTQKWFPAAINLVYTFAWNKTDIYGQKVWGLAEALVSVGYEYETCPDFILWEQRTVVLQGFEMDASNLGGWSLNKHHILNPQSGIIHKGNGENMFISQQPPVISTIMGNGHQRSVACTNCNGPAHNNKLFAPVALASGPDGSVYVGDFNFVRRIFPSGNSVSILELSTSPAHKYYLAMDPVSESLYLSDTNTRKVYKLKSLVETKDLSKNFEVVAGTGDQCLPFDQSHCGDGGRASEASLNSPRGITVDRHGFIYFVDGTMIRKIDENAVITTVIGSNGLTSTQPLSCDSGMDITQVRLEWPTDLAVNPMDNSLYVLDNNIVLQISENRRVRIIAGRPIHCQVPGIDHFLVSKVAIHSTLESARAISVSHSGLLFIAETDERKVNRIQQVTTNGEIYIIAGAPTDCDCKIDPNCDCFSGDGGYAKDAKMKAPSSLAVSPDGTLYVADLGNVRIRTISRNQAHLNDMNIYEIASPADQELYQFTVNGTHLHTLNLITRDYVYNFTYNSEGDLGAITSSNGNSVHIRRDAGGMPLWLVVPGGQVYWLTISSNGVLKRVSAQGYNLALMTYPGNTGLLATKSNENGWTTVYEYDPEGHLTNATFPTGEVSSFHSDLEKLTKVELDTSNRENVLMSTNLTATSTIYILKQENTQSTYRVNPDGSLRVTFASGMEIGLSSEPHILAGAVNPTLGKCNISLPGEHNANLIEWRQRKEQNKGNVSAFERRLRAHNRNLLSIDFDHITRTGKIYDDHRKFTLRILYDQTGRPILWSPVSRYNEVNITYSPSGLVTFIQRGTWNEKMEYDQSGKIISRTWADGKIWSYTYLEKSVMLLLHSQRRYIFEYDQPDCLLSVTMPSMVRHSLQTMLSVGYYRNIYTPPDSSTSFIQDYSRDGRLLQTLHLGTGRRVLYKYTKQARLSEVLYDTTQVTLTYEESSGVIKTIHLMHDGFICTIRYRQTGPLIGRQIFRFSEEGLVNARFDYSYNNFRVTSMQAVINETPLPIDLYRYVDVSGRTEQFGKFSVINYDLNQVITTTVMKHTKIFSANGQVIEVQYEILKAIAYWMTIQYDNVGRMVICDIRVGVDANITRYFYEYDADGQLQTVSVNDKTQWRYSYDLNGNINLLSHGKSARLTPLRYDLRDRITRLGEIQYKMDEDGFLRQRGNDIFEYNSNGLLQKAYNKASGWTVQYYYDGLGRRVASKSSLGQHLQFFYADLTNPIRVTHLYNHTSSEITSLYYDLQGHLIAMELSSGEEYYVACDNTGTPLAVFSSRGQVIKEILYTPYGDIYHDTYPDFQVIIGFHGGLYDFLTKLVHLGQRDYDVVAGRWTTPNHHIWKQLNLLPKPFNLYSFENNYPVGKIQDVAKYTTDIRSWLELFGFQLHNVLPGFPKPELENLELTYELLRLQTKTQEWDPGKTILGIQCELQKQLRNFISLDQLPMTPRYNDGRCLEGGKQPRFAAVPSVFGKGIKFAIKDGIVTADIIGVANEDSRRLAAILNNAHYLENLHFTIEGRDTHYFIKLGSLEEDLVLIGNTGGRRILENGVNVTVSQMTSVLNGRTRRFADIQLQHGALCFNIRYGTTVEEEKNHVLEIARQRAVAQAWTKEQRRLQEGEEGIRAWTEGEKQQLLSTGRVQGYDGYFVLSVEQYLELSDSANNIHFMRQSEIGRR",
# RAB21_HUMAN
"MAAAGGGGGGAAAAGRAYSFKVVLLGEGCVGKTSLVLRYCENKFNDKHITTLQASFLTKKLNIGGKRVNLAIWDTAGQERFHALGPIYYRDSNGAILVYDITDEDSFQKVKNWVKELRKMLGNEICLCIVGNKIDLEKERHVSIQEAESYAESVGAKHYHTSAKQNKGIEELFLDLCKRMIETAQVDERAKGNGSSQPGTARRGVQIIDDEPQAQTSGGGCCSSG",
# BL1S6_HUMAN
"MSVPGPSSPDGALTRPPYCLEAGEPTPGLSDTSPDEGLIEDLTIEDKAVEQLAEGLLSHYLPDLQRSKQALQELTQNQVVLLDTLEQEISKFKECHSMLDINALFAEAKHYHAKLVNIRKEMLMLHEKTSKLKKRALKLQQKRQKEELEREQQREKEFEREKQLTARPAKRM",
# TAOK2_HUMAN
"MPAGGRAGSLKDPDVAELFFKDDPEKLFSDLREIGHGSFGAVYFARDVRNSEVVAIKKMSYSGKQSNEKWQDIIKEVRFLQKLRHPNTIQYRGCYLREHTAWLVMEYCLGSASDLLEVHKKPLQEVEIAAVTHGALQGLAYLHSHNMIHRDVKAGNILLSEPGLVKLGDFGSASIMAPANSFVGTPYWMAPEVILAMDEGQYDGKVDVWSLGITCIELAERKPPLFNMNAMSALYHIAQNESPVLQSGHWSEYFRNFVDSCLQKIPQDRPTSEVLLKHRFVLRERPPTVIMDLIQRTKDAVRELDNLQYRKMKKILFQEAPNGPGAEAPEEEEEAEPYMHRAGTLTSLESSHSVPSMSISASSQSSSVNSLADASDNEEEEEEEEEEEEEEEGPEAREMAMMQEGEHTVTSHSSIIHRLPGSDNLYDDPYQPEITPSPLQPPAAPAPTSTTSSARRRAYCRNRDHFATIRTASLVSRQIQEHEQDSALREQLSGYKRMRRQHQKQLLALESRLRGEREEHSARLQRELEAQRAGFGAEAEKLARRHQAIGEKEARAAQAEERKFQQHILGQQKKELAALLEAQKRTYKLRKEQLKEELQENPSTPKREKAEWLLRQKEQLQQCQAEEEAGLLRRQRQYFELQCRQYKRKMLLARHSLDQDLLREDLNKKQTQKDLECALLLRQHEATRELELRQLQAVQRTRAELTRLQHQTELGNQLEYNKRREQELRQKHAAQVRQQPKSLKVRAGQRPPGLPLPIPGALGPPNTGTPIEQQPCSPGQEAVLDQRMLGEEEEAVGERRILGKEGATLEPKQQRILGEESGAPSPSPQKHGSLVDEEVWGLPEEIEELRVPSLVPQERSIVGQEEAGTWSLWGKEDESLLDEEFELGWVQGPALTPVPEEEEEEEEGAPIGTPRDPGDGCPSPDIPPEPPPTHLRPCPASQLPGLLSHGLLAGLSFAVGSSSGLLPLLLLLLLPLLAAQGGGGLQAALLALEVGLVGLGASYLLLCTALHLPSSLFLLLAQGTALGAVLGLSWRRGLMGVPLGLGAAWLLAWPGLALPLVAMAAGGRWVRQQGPRVRRGISRLWLRVLLRLSPMAFRALQGCGAVGDRGLFALYPKTNKDGFRSRLPVPGPRRRNPRTTQHPLALLARVWVLCKGWNWRLARASQGLASHLPPWAIHTLASWGLLRGERPTRIPRLLPRSQRQLGPPASRQPLPGTLAGRRSRTRQSRALPPWR",
# TRPC5_HUMAN
"MAQLYYKKVNYSPYRDRIPLQIVRAETELSAEEKAFLNAVEKGDYATVKQALQEAEIYYNVNINCMDPLGRSALLIAIENENLEIMELLLNHSVYVGDALLYAIRKEVVGAVELLLSYRRPSGEKQVPTLMMDTQFSEFTPDITPIMLAAHTNNYEIIKLLVQKRVTIPRPHQIRCNCVECVSSSEVDSLRHSRSRLNIYKALASPSLIALSSEDPILTAFRLGWELKELSKVENEFKAEYEELSQQCKLFAKDLLDQARSSRELEIILNHRDDHSEELDPQKYHDLAKLKVAIKYHQKEFVAQPNCQQLLATLWYDGFPGWRRKHWVVKLLTCMTIGFLFPMLSIAYLISPRSNLGLFIKKPFIKFICHTASYLTFLFMLLLASQHIVRTDLHVQGPPPTVVEWMILPWVLGFIWGEIKEMWDGGFTEYIHDWWNLMDFAMNSLYLATISLKIVAYVKYNGSRPREEWEMWHPTLIAEALFAISNILSSLRLISLFTANSHLGPLQISLGRMLLDILKFLFIYCLVLLAFANGLNQLYFYYETRAIDEPNNCKGIRCEKQNNAFSTLFETLQSLFWSVFGLLNLYVTNVKARHEFTEFVGATMFGTYNVISLVVLLNMLIAMMNNSYQLIADHADIEWKFARTKLWMSYFDEGGTLPPPFNIIPSPKSFLYLGNWFNNTFCPKRDPDGRRRRRNLRSFTERNADSLIQNQHYQEVIRNLVKRYVAAMIRNSKTHEGLTEENFKELKQDISSFRYEVLDLLGNRKHPRSFSTSSTELSQRDDNNDGSGGARAKSKSVSFNLGCKKKTCHGPPLIRTMPRSSGAQGKSKAESSSKRSFMGPSLKKLGLLFSKFNGHMSEPSSEPMYTISDGIVQQHCMWQDIRYSQMEKGKAEACSQSEINLSEVELGEVQGAAQSSECPLACSSSLHCASSICSSNSKLLDSSEDVFETWGEACDLLMHKWGDGQEEQVTTRL",
# MKLN1_HUMAN
"MAAGGAVAAAPECRLLPYALHKWSSFSSTYLPENILVDKPNDQSSRWSSESNYPPQYLILKLERPAIVQNITFGKYEKTHVCNLKKFKVFGGMNEENMTELLSSGLKNDYNKETFTLKHKIDEQMFPCRFIKIVPLLSWGPSFNFSIWYVELSGIDDPDIVQPCLNWYSKYREQEAIRLCLKHFRQHNYTEAFESLQKKTKIALEHPMLTDIHDKLVLKGDFDACEELIEKAVNDGLFNQYISQQEYKPRWSQIIPKSTKGDGEDNRPGMRGGHQMVIDVQTETVYLFGGWDGTQDLADFWAYSVKENQWTCISRDTEKENGPSARSCHKMCIDIQRRQIYTLGRYLDSSVRNSKSLKSDFYRYDIDTNTWMLLSEDTAADGGPKLVFDHQMCMDSEKHMIYTFGGRILTCNGSVDDSRASEPQFSGLFAFNCQCQTWKLLREDSCNAGPEDIQSRIGHCMLFHSKNRCLYVFGGQRSKTYLNDFFSYDVDSDHVDIISDGTKKDSGMVPMTGFTQRATIDPELNEIHVLSGLSKDKEKREENVRNSFWIYDIVRNSWSCVYKNDQAAKDNPTKSLQEEEPCPRFAHQLVYDELHKVHYLFGGNPGKSCSPKMRLDDFWSLKLCRPSKDYLLRHCKYLIRKHRFEEKAQVDPLSALKYLQNDLYITVDHSDPEETKEFQLLASALFKSGSDFTALGFSDVDHTYAQRTQLFDTLVNFFPDSMTPPKGNLVDLITL",
# NRX1A_HUMAN
"MGTALLQRGGCFLLCLSLLLLGCWAELGSGLEFPGAEGQWTRFPKWNACCESEMSFQLKTRSARGLVLYFDDEGFCDFLELILTRGGRLQLSFSIFCAEPATLLADTPVNDGAWHSVRIRRQFRNTTLFIDQVEAKWVEVKSKRRDMTVFSGLFVGGLPPELRAAALKLTLASVREREPFKGWIRDVRVNSSQVLPVDSGEVKLDDEPPNSGGGSPCEAGEEGEGGVCLNGGVCSVVDDQAVCDCSRTGFRGKDCSQEDNNVEGLAHLMMGDQGKSKGKEEYIATFKGSEYFCYDLSQNPIQSSSDEITLSFKTLQRNGLMLHTGKSADYVNLALKNGAVSLVINLGSGAFEALVEPVNGKFNDNAWHDVKVTRNLRQHSGIGHAMVTISVDGILTTTGYTQEDYTMLGSDDFFYVGGSPSTADLPGSPVSNNFMGCLKEVVYKNNDVRLELSRLAKQGDPKMKIHGVVAFKCENVATLDPITFETPESFISLPKWNAKKTGSISFDFRTTEPNGLILFSHGKPRHQKDAKHPQMIKVDFFAIEMLDGHLYLLLDMGSGTIKIKALLKKVNDGEWYHVDFQRDGRSGTISVNTLRTPYTAPGESEILDLDDELYLGGLPENKAGLVFPTEVWTALLNYGYVGCIRDLFIDGQSKDIRQMAEVQSTAGVKPSCSKETAKPCLSNPCKNNGMCRDGWNRYVCDCSGTGYLGRSCEREATVLSYDGSMFMKIQLPVVMHTEAEDVSLRFRSQRAYGILMATTSRDSADTLRLELDAGRVKLTVNLDCIRINCNSSKGPETLFAGYNLNDNEWHTVRVVRRGKSLKLTVDDQQAMTGQMAGDHTRLEFHNIETGIITERRYLSSVPSNFIGHLQSLTFNGMAYIDLCKNGDIDYCELNARFGFRNIIADPVTFKTKSSYVALATLQAYTSMHLFFQFKTTSLDGLILYNSGDGNDFIVVELVKGYLHYVFDLGNGANLIKGSSNKPLNDNQWHNVMISRDTSNLHTVKIDTKITTQITAGARNLDLKSDLYIGGVAKETYKSLPKLVHAKEGFQGCLASVDLNGRLPDLISDALFCNGQIERGCEGPSTTCQEDSCSNQGVCLQQWDGFSCDCSMTSFSGPLCNDPGTTYIFSKGGGQITYKWPPNDRPSTRADRLAIGFSTVQKEAVLVRVDSSSGLGDYLELHIHQGKIGVKFNVGTDDIAIEESNAIINDGKYHVVRFTRSGGNATLQVDSWPVIERYPAGRQLTIFNSQATIIIGGKEQGQPFQGQLSGLYYNGLKVLNMAAENDANIAIVGNVRLVGEVPSSMTTESTATAMQSEMSTSIMETTTTLATSTARRGKPPTKEPISQTTDDILVASAECPSDDEDIDPCEPSSGGLANPTRAGGREPYPGSAEVIRESSSTTGMVVGIVAAAALCILILLYAMYKYRNRDEGSYHVDESRNYISNSAQSNGAVVKEKQPSSAKSSNKNKKNKDKEYYV",
# CADH9_HUMAN
"MRTYHYIPLFIWTYMFHTVDTILLQEKPNSYLSSKKIAGLTKDDGKMLRRTKRGWMWNQFFLLEEYTGTDTQYVGKLHTDQDKGDGNLKYILTGDGAGSLFVIDENTGDIHAAKKLDREEKSLYILRAKAIDRKTGRQVEPESEFIIKIHDINDNEPKFTKDLYTASVPEMSGVGTSVIQVTATDADDANYGNSAKVVYSILQGQPYFSVDPESGIIKTALPDMSRENREQYQVVIQAKDMGGQMGGLSGTTTVNITLTDVNNNPPRFPQSTYQFNSPESVPLGTHLGRIKANDPDVGENAEMEYSIAEGDGADMFDVITDKDTQEGIITVKQNLDFENQMLYTLRVDASNTHPDPRFLHLGPFKDTAVVKISVEDIDEPPVFTKVSYLIEVDEDVKEGSIIGQVTAYDPDARNNLIKYSVDRHTDMDRIFGIHSENGSIFTLKALDRESSPWHNITVTATEINNPKQSSHIPVFIRILDINDHAPEFAMYYETFVCENAKPGQLIQTVSVMDKDDPPRGHKFFFEPVPEFTLNPNFTIVDNKDNTAGIMTRKDGYSRNKMSTYLLPILIFDNDYPIQSSTGTLTIRVCACDNQGNMQSCTAEALILSAGLSTGALVAILLCVLILLILVVLFAALKRQRKKEPLIISKDDVRDNIVTYNDEGGGEEDTQAFDIGTLRNPEAREDSKLRRDVMPETIFQIRRTVPLWENIDVQDFIHRRLKENDADPSAPPYDSLATYAYEGNDSIADSLSSLESLTADCNQDYDYLSDWGPRFKKLADMYGGDDSDRD",
# ZDHC8_HUMAN
"MPRSPGTRLKPAKYIPVATAAALLVGSSTLFFVFTCPWLTRAVSPAVPVYNGIIFLFVLANFSMATFMDPGVFPRADEDEDKEDDFRAPLYKNVDVRGIQVRMKWCATCHFYRPPRCSHCSVCDNCVEDFDHHCPWVNNCIGRRNYRYFFLFLLSLSAHMVGVVAFGLVYVLNHAEGLGAAHTTITMAVMCVAGLFFIPVIGLTGFHVVLVTRGRTTNEQVTGKFRGGVNPFTRGCCGNVEHVLCSPLAPRYVVEPPRLPLAVSLKPPFLRPELLDRAAPLKVKLSDNGLKAGLGRSKSKGSLDRLDEKPLDLGPPLPPKIEAGTFSSDLQTPRPGSAESALSVQRTSPPTPAMYKFRPAFPTGPKVPFCGPGEQVPGPDSLTLGDDSIRSLDFVSEPSLDLPDYGPGGLHAAYPPSPPLSASDAFSGALRSLSLKASSRRGGDHVALQPLRSEGGPPTPHRSIFAPHALPNRNGSLSYDSLLNPGSPGGHACPAHPAVGVAGYHSPYLHPGATGDPPRPLPRSFSPVLGPRPREPSPVRYDNLSRTIMASIQERKDREERERLLRSQADSLFGDSGVYDAPSSYSLQQASVLSEGPRGPALRYGSRDDLVAGPGFGGARNPALQTSLSSLSSSVSRAPRTSSSSLQADQASSNAPGPRPSSGSHRSPARQGLPSPPGTPHSPSYAGPKAVAFIHTDLPEPPPSLTVQRDHPQLKTPPSKLNGQSPGLARLGPATGPPGPSASPTRHTLVKKVSGVGGTTYEISV",
# KDIS_HUMAN
"MSVLISQSVINYVEEENIPALKALLEKCKDVDERNECGQTPLMIAAEQGNLEIVKELIKNGANCNLEDLDNWTALISASKEGHVHIVEELLKCGVNLEHRDMGGWTALMWACYKGRTDVVELLLSHGANPSVTGLYSVYPIIWAAGRGHADIVHLLLQNGAKVNCSDKYGTTPLVWAARKGHLECVKHLLAMGADVDQEGANSMTALIVAVKGGYTQSVKEILKRNPNVNLTDKDGNTALMIASKEGHTEIVQDLLDAGTYVNIPDRSGDTVLIGAVRGGHVEIVRALLQKYADIDIRGQDNKTALYWAVEKGNATMVRDILQCNPDTEICTKDGETPLIKATKMRNIEVVELLLDKGAKVSAVDKKGDTPLHIAIRGRSRKLAELLLRNPKDGRLLYRPNKAGETPYNIDCSHQKSILTQIFGARHLSPTETDGDMLGYDLYSSALADILSEPTMQPPICVGLYAQWGSGKSFLLKKLEDEMKTFAGQQIEPLFQFSWLIVFLTLLLCGGLGLLFAFTVHPNLGIAVSLSFLALLYIFFIVIYFGGRREGESWNWAWVLSTRLARHIGYLELLLKLMFVNPPELPEQTTKALPVRFLFTDYNRLSSVGGETSLAEMIATLSDACEREFGFLATRLFRVFKTEDTQGKKKWKKTCCLPSFVIFLFIIGCIISGITLLAIFRVDPKHLTVNAVLISIASVVGLAFVLNCRTWWQVLDSLLNSQRKRLHNAASKLHKLKSEGFMKVLKCEVELMARMAKTIDSFTQNQTRLVVIIDGLDACEQDKVLQMLDTVRVLFSKGPFIAIFASDPHIIIKAINQNLNSVLRDSNINGHDYMRNIVHLPVFLNSRGLSNARKFLVTSATNGDVPCSDTTGIQEDADRRVSQNSLGEMTKLGSKTALNRRDTYRRRQMQRTITRQMSFDLTKLLVTEDWFSDISPQTMRRLLNIVSVTGRLLRANQISFNWDRLASWINLTEQWPYRTSWLILYLEETEGIPDQMTLKTIYERISKNIPTTKDVEPLLEIDGDIRNFEVFLSSRTPVLVARDVKVFLPCTVNLDPKLREIIADVRAAREQISIGGLAYPPLPLHEGPPRAPSGYSQPPSVCSSTSFNGPFAGGVVSPQPHSSYYSGMTGPQHPFYNRPFFAPYLYTPRYYPGGSQHLISRPSVKTSLPRDQNNGLEVIKEDAAEGLSSPTDSSRGSGPAPGPVVLLNSLNVDAVCEKLKQIEGLDQSMLPQYCTTIKKANINGRVLAQCNIDELKKEMNMNFGDWHLFRSTVLEMRNAESHVVPEDPRFLSESSSGPAPHGEPARRASHNELPHTELSSQTPYTLNFSFEELNTLGLDEGAPRHSNLSWQSQTRRTPSLSSLNSQDSSIEISKLTDKVQAEYRDAYREYIAQMSQLEGGPGSTTISGRSSPHSTYYMGQSSSGGSIHSNLEQEKGKDSEPKPDDGRKSFLMKRGDVIDYSSSGVSTNDASPLDPITEEDEKSDQSGSKLLPGKKSSERSSLFQTDLKLKGSGLRYQKLPSDEDESGTEESDNTPLLKDDKDRKAEGKVERVPKSPEHSAEPIRTFIKAKEYLSDALLDKKDSSDSGVRSSESSPNHSLHNEVADDSQLEKANLIELEDDSHSGKRGIPHSLSGLQDPIIARMSICSEDKKSPSECSLIASSPEENWPACQKAYNLNRTPSTVTLNNNSAPANRANQNFDEMEGIRETSQVILRPSSSPNPTTIQNENLKSMTHKRSQRSSYTRLSKDPPELHAAASSESTGFGEERESIL",
# ASAP1_HUMAN
"MRSSASRLSSFSSRDSLWNRMPDQISVSEFIAETTEDYNSPTTSSFTTRLHNCRNTVTLLEEALDQDRTALQKVKKSVKAIYNSGQDHVQNEENYAQVLDKFGSNFLSRDNPDLGTAFVKFSTLTKELSTLLKNLLQGLSHNVIFTLDSLLKGDLKGVKGDLKKPFDKAWKDYETKFTKIEKEKREHAKQHGMIRTEITGAEIAEEMEKERRLFQLQMCEYLIKVNEIKTKKGVDLLQNLIKYYHAQCNFFQDGLKTADKLKQYIEKLAADLYNIKQTQDEEKKQLTALRDLIKSSLQLDQKEDSQSRQGGYSMHQLQGNKEYGSEKKGYLLKKSDGIRKVWQRRKCSVKNGILTISHATSNRQPAKLNLLTCQVKPNAEDKKSFDLISHNRTYHFQAEDEQDYVAWISVLTNSKEEALTMAFRGEQSAGENSLEDLTKAIIEDVQRLPGNDICCDCGSSEPTWLSTNLGILTCIECSGIHREMGVHISRIQSLELDKLGTSELLLAKNVGNNSFNDIMEANLPSPSPKPTPSSDMTVRKEYITAKYVDHRFSRKTCSTSSAKLNELLEAIKSRDLLALIQVYAEGVELMEPLLEPGQELGETALHLAVRTADQTSLHLVDFLVQNCGNLDKQTALGNTVLHYCSMYSKPECLKLLLRSKPTVDIVNQAGETALDIAKRLKATQCEDLLSQAKSGKFNPHVHVEYEWNLRQEEIDESDDDLDDKPSPIKKERSPRPQSFCHSSSISPQDKLALPGFSTPRDKQRLSYGAFTNQIFVSTSTDSPTSPTTEAPPLPPRNAGKGPTGPPSTLPLSTQTSSGSSTLSKKRPPPPPPGHKRTLSDPPSPLPHGPPNKGAVPWGNDGGPSSSSKTTNKFEGLSQQSSTSSAKTALGPRVLPKLPQKVALRKTDHLSLDKATIPPEIFQKSSQLAELPQKPPPGDLPPKPTELAPKPQIGDLPPKPGELPPKPQLGDLPPKPQLSDLPPKPQMKDLPPKPQLGDLLAKSQTGDVSPKAQQPSEVTLKSHPLDLSPNVQSRDAIQKQASEDSNDLTPTLPETPVPLPRKINTGKNKVRRVKTIYDCQADNDDELTFIEGEVIIVTGEEDQEWWIGHIEGQPERKGVFPVSFVHILSD",
# LRFN2_HUMAN
"METLLGGLLAFGMAFAVVDACPKYCVCQNLSESLGTLCPSKGLLFVPPDIDRRTVELRLGGNFIIHISRQDFANMTGLVDLTLSRNTISHIQPFSFLDLESLRSLHLDSNRLPSLGEDTLRGLVNLQHLIVNNNQLGGIADEAFEDFLLTLEDLDLSYNNLHGLPWDSVRRMVNLHQLSLDHNLLDHIAEGTFADLQKLARLDLTSNRLQKLPPDPIFARSQASALTATPFAPPLSFSFGGNPLHCNCELLWLRRLERDDDLETCGSPGGLKGRYFWHVREEEFVCEPPLITQHTHKLLVLEGQAATLKCKAIGDPSPLIHWVAPDDRLVGNSSRTAVYDNGTLDIFITTSQDSGAFTCIAANAAGEATAMVEVSIVQLPHLSNSTSRTAPPKSRLSDITGSSKTSRGGGGSGGGEPPKSPPERAVLVSEVTTTSALVKWSVSKSAPRVKMYQLQYNCSDDEVLIYRMIPASNKAFVVNNLVSGTGYDLCVLAMWDDTATTLTATNIVGCAQFFTKADYPQCQSMHSQILGGTMILVIGGIIVATLLVFIVILMVRYKVCNHEAPSKMAAAVSNVYSQTNGAQPPPPSSAPAGAPPQGPPKVVVRNELLDFTASLARASDSSSSSSLGSGEAAGLGRAPWRIPPSAPRPKPSLDRLMGAFASLDLKSQRKEELLDSRTPAGRGAGTSARGHHSDREPLLGPPAARARSLLPLPLEGKAKRSHSFDMGDFAAAAAGGVVPGGYSPPRKVSNIWTKRSLSVNGMLLPFEESDLVGARGTFGSSEWVMESTV",
# MRTFB_HUMAN
"MIDSSKKQQQGFPEILTAGDFEPLKEKECLEGSNQKSLKEVLQLRLQQRRTREQLVDQGIMPPLKSPAAFHEQIKSLERARTENFLKHKIRSRPDRSELVRMHILEETFAEPSLQATQMKLKRARLADDLNEKIAQRPGPMELVEKNILPVDSSVKEAIIGVGKEDYPHTQGDFSFDEDSSDALSPDQPASQESQGSAASPSEPKVSESPSPVTTNTPAQFASVSPTVPEFLKTPPTADQPPPRPAAPVLPTNTVSSAKPGPALVKQSHPKNPNDKHRSKKCKDPKPRVKKLKYHQYIPPDQKGEKNEPQMDSNYARLLQQQQLFLQLQILSQQKQHYNYQTILPAPFKPLNDKNSNSGNSALNNATPNTPRQNTSTPVRKPGPLPSSLDDLKVSELKTELKLRGLPVSGTKPDLIERLKPYQEVNSSGLAAGGIVAVSSSAIVTSNPEVTVALPVTTLHNTVTSSVSTLKAELPPTGTSNATRVENVHSPLPISPSPSEQSSLSTDDTNMADTFTEIMTMMSPSQFLSSSPLRMTNNEDSLSPTSSTLSNLELDAAEKDRKLQEKEKQIEELKRKLEQEQKLVEVLKMQLEVEKRGQQQRPLEAQPSAPGHSVKSDQKHGSLGSSIKDEASLPDCSSSRQPIPVASHAVGQPVSTGGQTLVAKKAVVIKQEVPVGQAEQQSVVSQFYVSSQGQPPPAVVAQPQALLTTQTAQLLLPVSIQGSSVTSVQLPVGSLKLQTSPQAGMQTQPQIATAAQIPTAALASGLAPTVPQTQDTFPQHVLSQPQQVRKVFTNSASSNTVLPYQRHPAPAVQQPFINKASNSVLQSRNAPLPSLQNGPNTPNKPSSPPPPQQFVVQHSLFGSPVAKTKDPPRYEEAIKQTRSTQAPLPEISNAHSQQMDDLFDILIKSGEISLPIKEEPSPISKMRPVTASITTMPVNTVVSRPPPQVQMAPPVSLEPMGSLSASLENQLEAFLDGTLPSANEIPPLQSSSEDREPFSLIEDLQNDLLSHSGMLDHSHSPMETSETQFAAGTPCLSLDLSDSNLDNMEWLDITMPNSSSGLTPLSTTAPSMFSADFLDPQDLPLPWD",
# NWD2_HUMAN
"MWPAGAGTKLPCPRDSALRRAAFSGNLTALPSHLVPAGRSVRVFISANPEDTGAERQALRENVYPKLREFCRENYGLEFQVIDLYWGVEEDEWDSPELQKTRMKLLENCLKTSAGPCFVGLLGEKYGNIRIPGEVEASEFEMILDAAIEAKLETKLLEEWYCRDENSVPAAYYLRPKSEMLRSNRNAMQPSTNAENEKTWQEISDEIKKIFKAAVKLLHEKGKMKHSQAKRYLFSAIEDEFDFALGKQTPAFLKKCVCYIRKIANIERFVKIPEMGKYMDITGTEPRIIRDPEAQEKLIKLRDEFIPTIVASSNLRVYTSVTHCDMKLGYSQEIENHYIEGLGKQFYEDMIDIIQATIQQNFDTETDTLYDEILQHSSLCKTYASFYEYKCESLNIVHNYILPSKAGHINPLIIYGGPCTGKTLLLAEVAKKAYGWLHEDTGPESDPVVIVRFLGTTDMSSDLRTLLLSVCEQLAVNYRCLVQSYPKKIHDLCDLFINLLNESSLQRPLVIIFDALEQLSENDDARKLWWLPAHLPRFVRIVLSTLPNKHGILQKLRCLIHEEDNYIELIPRDRKMCSQVLKHQLLRVKRKVTSGQQIYVNNALSKCTLPMFVNLTFREVRHWRSHKDVDESSLSVTVHESIEQLFWSLEKKCGQKLVSRALGYITMAKMGLSEMELEDVLALDNSVMSELKENTRPSNPLRVPYLYIARLKEGLSGYLIERHVKNVTLLVWANRHLQLIAQKLYLQDDNDLREMHTILADYFLGVWSGGRRKAFCLEDPYLNGCLDLENRSLLEEEKHFMEQASFDRQAPDQPWVFQCNPLEPDIFFVNHRKMSELLYHLTRCGKTDDLLYGIIMNFSWLYTMIKIGQFDKVLSDIELAYNYSQEKELKFLANTLRSIKNKVTAFPGSLSAELQQRLLPVVSSLPKLRHLLLECDKDGPKYCSIVPLHSSMDVTYSPERLPLSSSHLHVTEILPTCNPSTVLTALENGSISTWDVETRQLLRQITTAQSVILGMKLTSDEKYLVVATTNNTLLIYDNVNSCLLSEVEIKGTKHGSSATYINGFTLSANHALAWLEASKDVTVIDLLYGWPLYQFHCWYEVTCVQCSLDGLYAFCGQYLNTTTIFHLGSGEKLCTVTSEFSGGFVKFLLILDTAQEMVMVDSEGSLSVWNTEDISSPQLTDDFDCRREDSEVVSIELSEDQSAVLICKALSIELLDTGLWKVAEKFRAKHNERFISAVLSKNGDCIIATMENTSAVFFWRRDTGQCMASLQEISGSIVKLVKSSHHNMLLSLSTSGVLSIWDIDIITAMSNIDKTGKPIQSLLLPARGEIIYSLDGSDCVHKWNFSSGFIEAVFKHEGIVEHCVLTSTGDIMVTSDDKSSQYVWHTSSGENLFRINGQRISQLLITHNDQFVVSLCEENASRVWRLATGHRVCNILTTLQNAFITSANTFVVGMTKSKVLAVSLWTGSITKKFCCEDGTTIVNFKLIPDCPDIIVFITSAETVNIWSLTDEVICRRVQLPNNFLKNLEDFEISPNGKLGIIARGDENINVLDLYSGKLRVVHASGIIWRQRLSRDGRYLVYICFRNGEEEDENGAIFSLIVMRLADGKNIGACSLYKTPTFLALSQRHLNIIVGFDDGSIGIYTVVDRVDAALKIKIATSNSRQIFNNATHTSRPKCNSYCFKISVDCLWRESTEVFARDSPITVSDSTESNEATPSKKHNSCYERVCSALEARGHSYAPDN",
# NEB1_HUMAN
"MLKTESSGERTTLRSASPHRNAYRTEFQALKSTFDKPKSDGEQKTKEGEGSQQSRGRKYGSNVNRIKNLFMQMGMEPNENAAVIAKTRGKGGHSSPQRRMKPKEFLEKTDGSVVKLESSVSERISRFDTMYDGPSYSKFTETRKMFERSVHESGQNNRYSPKKEKAGGSEPQDEWGGSKSNRGSTDSLDSLSSRTEAVSPTVSQLSAVFENTDSPSAIISEKAENNEYSVTGHYPLNLPSVTVTNLDTFGHLKDSNSWPPSNKRGVDTEDAHKSNATPVPEVASKSTSLASIPGEEIQQSKEPEDSTSNQQTPDSIDKDGPEEPCAESKAMPKSEIPSPQSQLLEDAEANLVGREAAKQQRKELAGGDFTSPDASASSCGKEVPEDSNNFDGSHVYMHSDYNVYRVRSRYNSDWGETGTEQDEEEDSDENSYYQPDMEYSEIVGLPEEEEIPANRKIKFSSAPIKVFNTYSNEDYDRRNDEVDPVAASAEYELEKRVEKLELFPVELEKDEDGLGISIIGMGVGADAGLEKLGIFVKTVTEGGAAQRDGRIQVNDQIVEVDGISLVGVTQNFAATVLRNTKGNVRFVIGREKPGQVSEVAQLISQTLEQERRQRELLEQHYAQYDADDDETGEYATDEEEDEVGPVLPGSDMAIEVFELPENEDMFSPSELDTSKLSHKFKELQIKHAVTEAEIQKLKTKLQAAENEKVRWELEKTQLQQNIEENKERMLKLESYWIEAQTLCHTVNEHLKETQSQYQALEKKYNKAKKLIKDFQQKELDFIKRQEAERKKIEDLEKAHLVEVQGLQVRIRDLEAEVFRLLKQNGTQVNNNNNIFERRTSLGEVSKGDTMENLDGKQTSCQDGLSQDLNEAVPETERLDSKALKTRAQLSVKNRRQRPSRTRLYDSVSSTDGEDSLERKNFTFNDDFSPSSTSSADLSGLGAEPKTPGLSQSLALSSDESLDMIDDEILDDGQSPKHSQCQNRAVQEWSVQQVSHWLMSLNLEQYVSEFSAQNITGEQLLQLDGNKLKALGMTASQDRAVVKKKLKEMKMSLEKARKAQEKMEKQREKLRRKEQEQMQRKSKKTEKMTSTTAEGAGEQ",
# GRID1_HUMAN
"MEALTLWLLPWICQCVSVRADSIIHIGAIFEENAAKDDRVFQLAVSDLSLNDDILQSEKITYSIKVIEANNPFQAVQEACDLMTQGILALVTSTGCASANALQSLTDAMHIPHLFVQRNPGGSPRTACHLNPSPDGEAYTLASRPPVRLNDVMLRLVTELRWQKFVMFYDSEYDIRGLQSFLDQASRLGLDVSLQKVDKNISHVFTSLFTTMKTEELNRYRDTLRRAILLLSPQGAHSFINEAVETNLASKDSHWVFVNEEISDPEILDLVHSALGRMTVVRQIFPSAKDNQKCTRNNHRISSLLCDPQEGYLQMLQISNLYLYDSVLMLANAFHRKLEDRKWHSMASLNCIRKSTKPWNGGRSMLDTIKKGHITGLTGVMEFREDSSNPYVQFEILGTTYSETFGKDMRKLATWDSEKGLNGSLQERPMGSRLQGLTLKVVTVLEEPFVMVAENILGQPKRYKGFSIDVLDALAKALGFKYEIYQAPDGRYGHQLHNTSWNGMIGELISKRADLAISAITITPERESVVDFSKRYMDYSVGILIKKPEEKISIFSLFAPFDFAVWACIAAAIPVVGVLIFVLNRIQAVRAQSAAQPRPSASATLHSAIWIVYGAFVQQGGESSVNSMAMRIVMGSWWLFTLIVCSSYTANLAAFLTVSRMDNPIRTFQDLSKQVEMSYGTVRDSAVYEYFRAKGTNPLEQDSTFAELWRTISKNGGADNCVSSPSEGIRKAKKGNYAFLWDVAVVEYAALTDDDCSVTVIGNSISSKGYGIALQHGSPYRDLFSQRILELQDTGDLDVLKQKWWPHMGRCDLTSHASAQADGKSLKLHSFAGVFCILAIGLLLACLVAALELWWNSNRCHQETPKEDKEVNLEQVHRRMNSLMDEDIAHKQISPASIELSALEMGGLAPTQTLEPTREYQNTQLSVSTFLPEQSSHGTSRTLSSGPSSNLPLPLSSSATMPSMQCKHRSPNGGLFRQSPVKTPIPMSFQPVPGGVLPEALDTSHGTSI",
# VANG2_HUMAN
"MDTESQYSGYSYKSGHSRSSRKHRDRRDRHRSKSRDGGRGDKSVTIQAPGEPLLDNESTRGDERDDNWGETTTVVTGTSEHSISHDDLTRIAKDMEDSVPLDCSRHLGVAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAFKLLILLLGSWALFFRRPKASLPRVFVLRALLMVLVFLLVVSYWLFYGVRILDARERSYQGVVQFAVSLVDALLFVHYLAVVLLELRQLQPQFTLKVVRSTDGASRFYNVGHLSIQRVAVWILEKYYHDFPVYNPALLNLPKSVLAKKVSGFKVYSLGEENSTNNSTGQSRAVIAAAARRRDNSHNEYYYEEAEHERRVRKRRARLVVAVEEAFTHIKRLQEEEQKNPREVMDPREAAQAIFASMARAMQKYLRTTKQQPYHTMESILQHLEFCITHDMTPKAFLERYLAAGPTIQYHKERWLAKQWTLVSEEPVTNGLKDGIVFLLKRQDFSLVVSTKKVPFFKLSEEFVDPKSHKFVMRLQSETSV",
# SHRM4_HUMAN
"MENRPGSFQYVPVQLQGGAPWGFTLKGGLEHCEPLTVSKIEDGGKAALSQKMRTGDELVNINGTPLYGSRQEALILIKGSFRILKLIVRRRNAPVSRPHSWHVAKLLEGCPEAATTMHFPSEAFSLSWHSGCNTSDVCVQWCPLSRHCSTEKSSSIGSMESLEQPGQATYESHLLPIDQNMYPNQRDSAYSSFSASSNASDCALSLRPEEPASTDCIMQGPGPTKAPSGRPNVAETSGGSRRTNGGHLTPSSQMSSRPQEGYQSGPAKAVRGPPQPPVRRDSLQASRAQLLNGEQRRASEPVVPLPQKEKLSLEPVLPARNPNRFCCLSGHDQVTSEGHQNCEFSQPPESSQQGSEHLLMQASTKAVGSPKACDRASSVDSNPLNEASAELAKASFGRPPHLIGPTGHRHSAPEQLLASHLQHVHLDTRGSKGMELPPVQDGHQWTLSPLHSSHKGKKSPCPPTGGTHDQSSKERKTRQVDDRSLVLGHQSQSSPPHGEADGHPSEKGFLDPNRTSRAASELANQQPSASGSLVQQATDCSSTTKAASGTEAGEEGDSEPKECSRMGGRRSGGTRGRSIQNRRKSERFATNLRNEIQRRKAQLQKSKGPLSQLCDTKEPVEETQEPPESPPLTASNTSLLSSCKKPPSPRDKLFNKSMMLRARSSECLSQAPESHESRTGLEGRISPGQRPGQSSLGLNTWWKAPDPSSSDPEKAHAHCGVRGGHWRWSPEHNSQPLVAAAMEGPSNPGDNKELKASTAQAGEDAILLPFADRRKFFEESSKSLSTSHLPGLTTHSNKTFTQRPKPIDQNFQPMSSSCRELRRHPMDQSYHSADQPYHATDQSYHSMSPLQSETPTYSECFASKGLENSMCCKPLHCGDFDYHRTCSYSCSVQGALVHDPCIYCSGEICPALLKRNMMPNCYNCRCHHHQCIRCSVCYHNPQHSALEDSSLAPGNTWKPRKLTVQEFPGDKWNPITGNRKTSQSGREMAHSKTSFSWATPFHPCLENPALDLSSYRAISSLDLLGDFKHALKKSEETSVYEEGSSLASMPHPLRSRAFSESHISLAPQSTRAWGQHRRELFSKGDETQSDLLGARKKAFPPPRPPPPNWEKYRLFRAAQQQKQQQQQQKQQEEEEEEEEEEEEEEEEEEEEAEEEEEELPPQYFSSETSGSCALNPEEVLEQPQPLSFGHLEGSRQGSQSVPAEQESFALHSSDFLPPIRGHLGSQPEQAQPPCYYGIGGLWRTSGQEATESAKQEFQHFSPPSGAPGIPTSYSAYYNISVAKAELLNKLKDQPEMAEIGLGEEEVDHELAQKKIQLIESISRKLSVLREAQRGLLEDINANSALGEEVEANLKAVCKSNEFEKYHLFVGDLDKVVNLLLSLSGRLARVENALNSIDSEANQEKLVLIEKKQQLTGQLADAKELKEHVDRREKLVFGMVSRYLPQDQLQDYQHFVKMKSALIIEQRELEEKIKLGEEQLKCLRESLLLGPSNF",
# TBC24_HUMAN
"MDSPGYNCFVDKDKMDAAIQDLGPKELSCTELQELKQLARQGYWAQSHALRGKVYQRLIRDIPCRTVTPDASVYSDIVGKIVGKHSSSCLPLPEFVDNTQVPSYCLNARGEGAVRKILLCLANQFPDISFCPALPAVVALLLHYSIDEAECFEKACRILACNDPGRRLIDQSFLAFESSCMTFGDLVNKYCQAAHKLMVAVSEDVLQVYADWQRWLFGELPLCYFARVFDVFLVEGYKVLYRVALAILKFFHKVRAGQPLESDSVKQDIRTFVRDIAKTVSPEKLLEKAFAIRLFSRKEIQLLQMANEKALKQKGITVKQKSVSLSKRQFVHLAVHAENFRSEIVSVREMRDIWSWVPERFALCQPLLLFSSLQHGYSLARFYFQCEGHEPTLLLIKTTQKEVCGAYLSTDWSERNKFGGKLGFFGTGECFVFRLQPEVQRYEWVVIKHPELTKPPPLMAAEPTAPLSHSASSDPADRLSPFLAARHFNLPSKTESMFMAGGSDCLIVGGGGGQALYIDGDLNRGRTSHCDTFNNQPLCSENFLIAAVEAWGFQDPDTQ",
# PPM1H_HUMAN
"MLTRVKSAVANFMGGIMAGSSGSEHGGGSCGGSDLPLRFPYGRPEFLGLSQDEVECSADHIARPILILKETRRLPWATGYAEVINAGKSTHNEDQASCEVLTVKKKAGAVTSTPNRNSSKRRSSLPNGEGLQLKENSESEGVSCHYWSLFDGHAGSGAAVVASRLLQHHITEQLQDIVDILKNSAVLPPTCLGEEPENTPANSRTLTRAASLRGGVGAPGSPSTPPTRFFTEKKIPHECLVIGALESAFKEMDLQIERERSSYNISGGCTALIVICLLGKLYVANAGDSRAIIIRNGEIIPMSSEFTPETERQRLQYLAFMQPHLLGNEFTHLEFPRRVQRKELGKKMLYRDFNMTGWAYKTIEDEDLKFPLIYGEGKKARVMATIGVTRGLGDHDLKVHDSNIYIKPFLSSAPEVRIYDLSKYDHGSDDVLILATDGLWDVLSNEEVAEAITQFLPNCDPDDPHRYTLAAQDLVMRARGVLKDRGWRISNDRLGSGDDISVYVIPLIHGNKLS",
# CAPS1_HUMAN
"MLDPSSSEEESDEIVEEESGKEVLGSAPSGARLSPSRTSEGSAGSAGLGGGGAGAGAGVGAGGGGGSGASSGGGAGGLQPSSRAGGGRPSSPSPSVVSEKEKEELERLQKEEEERKKRLQLYVFVMRCIAYPFNAKQPTDMARRQQKISKQQLQTVKDRFQAFLNGETQIMADEAFMNAVQSYYEVFLKSDRVARMVQSGGCSANDSREVFKKHIEKRVRSLPEIDGLSKETVLSSWMAKFDAIYRGEEDPRKQQARMTASAASELILSKEQLYEMFQNILGIKKFEHQLLYNACQLDNPDEQAAQIRRELDGRLQMADQIARERKFPKFVSKEMENMYIEELKSSVNLLMANLESMPVSKGGEFKLQKLKRSHNASIIDMGEESENQLSKSDVVLSFSLEVVIMEVQGLKSLAPNRIVYCTMEVEGGEKLQTDQAEASKPTWGTQGDFSTTHALPAVKVKLFTESTGVLALEDKELGRVILHPTPNSPKQSEWHKMTVSKNCPDQDLKIKLAVRMDKPQNMKHSGYLWAIGKNVWKRWKKRFFVLVQVSQYTFAMCSYREKKAEPQELLQLDGYTVDYTDPQPGLEGGRAFFNAVKEGDTVIFASDDEQDRILWVQAMYRATGQSHKPVPPTQVQKLNAKGGNVPQLDAPISQFYADRAQKHGMDEFISSNPCNFDHASLFEMVQRLTLDHRLNDSYSCLGWFSPGQVFVLDEYCARNGVRGCHRHLCYLRDLLERAENGAMIDPTLLHYSFAFCASHVHGNRPDGIGTVTVEEKERFEEIKERLRVLLENQITHFRYCFPFGRPEGALKATLSLLERVLMKDIVTPVPQEEVKTVIRKCLEQAALVNYSRLSEYAKIEENQKDAENVGRLITPAKKLEDTIRLAELVIEVLQQNEEHHAEPHVDKGEAFAWWSDLMVEHAETFLSLFAVDMDAALEVQPPDTWDSFPLFQLLNDFLRTDYNLCNGKFHKHLQDLFAPLVVRYVDLMESSIAQSIHRGFERESWEPVKSLTSNLPNVNLPNVNLPKVPNLPVNIPLGIPQMPTFSAPSWMAAIYDADNGSGTSEDLFWKLDALQTFIRDLHWPEEEFGKHLEQRLKLMASDMIESCVKRTRIAFEVKLQKTSRSTDFRVPQSICTMFNVMVDAKAQSTKLCSMEMGQEHQYHSKIDELIEETVKEMITLLVAKFVTILEGVLAKLSRYDEGTLFSSFLSFTVKAASKYVDVPKPGMDVADAYVTFVRHSQDVLRDKVNEEMYIERLFDQWYNSSMNVICTWLTDRMDLQLHIYQLKTLIRMVKKTYRDFRLQGVLDSTLNSKTYETIRNRLTVEEATASVSEGGGLQGISMKDSDEEDEEDD",
# MYO5B_HUMAN
"MSVGELYSQCTRVWIPDPDEVWRSAELTKDYKEGDKSLQLRLEDETILEYPIDVQRNQLPFLRNPDILVGENDLTALSYLHEPAVLHNLKVRFLESNHIYTYCGIVLVAINPYEQLPIYGQDVIYTYSGQNMGDMDPHIFAVAEEAYKQMARDEKNQSIIVSGESGAGKTVSAKYAMRYFATVGGSASETNIEEKVLASSPIMEAIGNAKTTRNDNSSRFGKYIQIGFDKRYHIIGANMRTYLLEKSRVVFQADDERNYHIFYQLCAAAGLPEFKELALTSAEDFFYTSQGGDTSIEGVDDAEDFEKTRQAFTLLGVKESHQMSIFKIIASILHLGSVAIQAERDGDSCSISPQDVYLSNFCRLLGVEHSQMEHWLCHRKLVTTSETYVKTMSLQQVINARNALAKHIYAQLFGWIVEHINKALHTSLKQHSFIGVLDIYGFETFEVNSFEQFCINYANEKLQQQFNSHVFKLEQEEYMKEQIPWTLIDFYDNQPCIDLIEAKLGILDLLDEECKVPKGTDQNWAQKLYDRHSSSQHFQKPRMSNTAFIIVHFADKVEYLSDGFLEKNRDTVYEEQINILKASKFPLVADLFHDDKDPVPATTPGKGSSSKISVRSARPPMKVSNKEHKKTVGHQFRTSLHLLMETLNATTPHYVRCIKPNDEKLPFHFDPKRAVQQLRACGVLETIRISAAGYPSRWAYHDFFNRYRVLVKKRELANTDKKAICRSVLENLIKDPDKFQFGRTKIFFRAGQVAYLEKLRADKFRTATIMIQKTVRGWLQKVKYHRLKGATLTLQRYCRGHLARRLAEHLRRIRAAVVLQKHYRMQRARQAYQRVRRAAVVIQAFTRAMFVRRTYRQVLMEHKATTIQKHVRGWMARRHFQRLRDAAIVIQCAFRMLKARRELKALRIEARSAEHLKRLNVGMENKVVQLQRKIDEQNKEFKTLSEQLSVTTSTYTMEVERLKKELVHYQQSPGEDTSLRLQEEVESLRTELQRAHSERKILEDAHSREKDELRKRVADLEQENALLKDEKEQLNNQILCQSKDEFAQNSVKENLMKKELEEERSRYQNLVKEYSQLEQRYDNLRDEMTIIKQTPGHRRNPSNQSSLESDSNYPSISTSEIGDTEDALQQVEEIGLEKAAMDMTVFLKLQKRVRELEQERKKLQVQLEKREQQDSKKVQAEPPQTDIDLDPNADLAYNSLKRQELESENKKLKNDLNELRKAVADQATQNNSSHGSPDSYSLLLNQLKLAHEELEVRKEEVLILRTQIVSADQRRLAGRNAEPNINARSSWPNSEKHVDQEDAIEAYHGVCQTNSKTEDWGYLNEDGELGLAYQGLKQVARLLEAQLQAQSLEHEEEVEHLKAQLEALKEEMDKQQQTFCQTLLLSPEAQVEFGVQQEISRLTNENLDLKELVEKLEKNERKLKKQLKIYMKKAQDLEAAQALAQSERKRHELNRQVTVQRKEKDFQGMLEYHKEDEALLIRNLVTDLKPQMLSGTVPCLPAYILYMCIRHADYTNDDLKVHSLLTSTINGIKKVLKKHNDDFEMTSFWLSNTCRLLHCLKQYSGDEGFMTQNTAKQNEHCLKNFDLTEYRQVLSDLSIQIYQQLIKIAEGVLQPMIVSAMLENESIQGLSGVKPTGYRKRSSSMADGDNSYCLEAIIRQMNAFHTVMCDQGLDPEIILQVFKQLFYMINAVTLNNLLLRKDVCSWSTGMQLRYNISQLEEWLRGRNLHQSGAVQTMEPLIQAAQLLQLKKKTQEDAEAICSLCTSLSTQQIVKILNLYTPLNEFEERVTVAFIRTIQAQLQERNDPQQLLLDAKHMFPVLFPFNPSSLTMDSIHIPACLNLEFLNEV",
# FZD4_HUMAN
"MAWRGAGPSVPGAPGGVGLSLGLLLQLLLLLGPARGFGDEEERRCDPIRISMCQNLGYNVTKMPNLVGHELQTDAELQLTTFTPLIQYGCSSQLQFFLCSVYVPMCTEKINIPIGPCGGMCLSVKRRCEPVLKEFGFAWPESLNCSKFPPQNDHNHMCMEGPGDEEVPLPHKTPIQPGEECHSVGTNSDQYIWVKRSLNCVLKCGYDAGLYSRSAKEFTDIWMAVWASLCFISTAFTVLTFLIDSSRFSYPERPIIFLSMCYNIYSIAYIVRLTVGRERISCDFEEAAEPVLIQEGLKNTGCAIIFLLMYFFGMASSIWWVILTLTWFLAAGLKWGHEAIEMHSSYFHIAAWAIPAVKTIVILIMRLVDADELTGLCYVGNQNLDALTGFVVAPLFTYLVIGTLFIAAGLVALFKIRSNLQKDGTKTDKLERLMVKIGVFSVLYTVPATCVIACYFYEISNWALFRYSADDSNMAVEMLKIFMSLLVGITSGMWIWSAKTLHTWQKCSNRLVNSGKVKREKRGNGWVKPGKGSETVV",
# RAB26_HUMAN
"MSRKKTPKSKGASTPAASTLPTANGARPARSGTALSGPDAPPNGPLQPGRPSLGGGVDFYDVAFKVMLVGDSGVGKTCLLVRFKDGAFLAGTFISTVGIDFRNKVLDVDGVKVKLQMWDTAGQERFRSVTHAYYRDAHALLLLYDVTNKASFDNIQAWLTEIHEYAQHDVALMLLGNKVDSAHERVVKREDGEKLAKEYGLPFMETSAKTGLNVDLAFTAIAKELKQRSMKAPSEPRFRLHDYVKREGRGASCCRP",
# FZR1_HUMAN
"MDQDYERRLLRQIVIQNENTMPRVTEMRRTLTPASSPVSSPSKHGDRFIPSRAGANWSVNFHRINENEKSPSQNRKAKDATSDNGKDGLAYSALLKNELLGAGIEKVQDPQTEDRRLQPSTPEKKGLFTYSLSTKRSSPDDGNDVSPYSLSPVSNKSQKLLRSPRKPTRKISKIPFKVLDAPELQDDFYLNLVDWSSLNVLSVGLGTCVYLWSACTSQVTRLCDLSVEGDSVTSVGWSERGNLVAVGTHKGFVQIWDAAAGKKLSMLEGHTARVGALAWNAEQLSSGSRDRMILQRDIRTPPLQSERRLQGHRQEVCGLKWSTDHQLLASGGNDNKLLVWNHSSLSPVQQYTEHLAAVKAIAWSPHQHGLLASGGGTADRCIRFWNTLTGQPLQCIDTGSQVCNLAWSKHANELVSTHGYSQNQILVWKYPSLTQVAKLTGHSYRVLYLAMSPDGEAIVTGAGDETLRFWNVFSKTRSTKVKWESVSVLNLFTRIR",
# MYO6_HUMAN
"MEDGKPVWAPHPTDGFQMGNIVDIGPDSLTIEPLNQKGKTFLALINQVFPAEEDSKKDVEDNCSLMYLNEATLLHNIKVRYSKDRIYTYVANILIAVNPYFDIPKIYSSEAIKSYQGKSLGTRPPHVFAIADKAFRDMKVLKMSQSIIVSGESGAGKTENTKFVLRYLTESYGTGQDIDDRIVEANPLLEAFGNAKTVRNNNSSRFGKFVEIHFNEKSSVVGGFVSHYLLEKSRICVQGKEERNYHIFYRLCAGASEDIREKLHLSSPDNFRYLNRGCTRYFANKETDKQILQNRKSPEYLKAGSMKDPLLDDHGDFIRMCTAMKKIGLDDEEKLDLFRVVAGVLHLGNIDFEEAGSTSGGCNLKNKSAQSLEYCAELLGLDQDDLRVSLTTRVMLTTAGGTKGTVIKVPLKVEQANNARDALAKTVYSHLFDHVVNRVNQCFPFETSSYFIGVLDIAGFEYFEHNSFEQFCINYCNEKLQQFFNERILKEEQELYQKEGLGVNEVHYVDNQDCIDLIEAKLVGILDILDEENRLPQPSDQHFTSAVHQKHKDHFRLTIPRKSKLAVHRNIRDDEGFIIRHFAGAVCYETTQFVEKNNDALHMSLESLICESRDKFIRELFESSTNNNKDTKQKAGKLSFISVGNKFKTQLNLLLDKLRSTGASFIRCIKPNLKMTSHHFEGAQILSQLQCSGMVSVLDLMQGGYPSRASFHELYNMYKKYMPDKLARLDPRLFCKALFKALGLNENDYKFGLTKVFFRPGKFAEFDQIMKSDPDHLAELVKRVNHWLTCSRWKKVQWCSLSVIKLKNKIKYRAEACIKMQKTIRMWLCKRRHKPRIDGLVKVGTLKKRLDKFNEVVSVLKDGKPEMNKQIKNLEISIDTLMAKIKSTMMTQEQIQKEYDALVKSSEELLSALQKKKQQEEEAERLRRIQEEMEKERKRREEDEKRRRKEEEERRMKLEMEAKRKQEEEERKKREDDEKRIQAEVEAQLARQKEEESQQQAVLEQERRDRELALRIAQSEAELISDEAQADLALRRSLDSYPVSKNDGTRPKMTPEQMAKEMSEFLSRGPAVLATKAAAGTKKYDLSKWKYAELRDTINTSCDIELLAACREEFHRRLKVYHAWKSKNKKRNTETEQRAPKSVTDYDFAPFLNNSPQQNPAAQIPARQREIEMNRQQRFFRIPFIRPADQYKDPQSKKKGWWYAHFDGPWIARQMELHPDKPPILLVAGKDDMEMCELNLEETGLTRKRGAEILPRQFEEIWERCGGIQYLQNAIESRQARPTYATAMLQSLLK",
# ALK_HUMAN
"MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSYSRLQRKSLAVDFVVPSLFRVYARDLLLPPSSSELKAGRPEARGSLALDCAPLLRLLGPAPGVSWTAGSPAPAEARTLSRVLKGGSVRKLRRAKQLVLELGEEAILEGCVGPPGEAAVGLLQFNLSELFSWWIRQGEGRLRIRLMPEKKASEVGREGRLSAAIRASQPRLLFQIFGTGHSSLESPTNMPSPSPDYFTWNLTWIMKDSFPFLSHRSRYGLECSFDFPCELEYSPPLHDLRNQSWSWRRIPSEEASQMDLLDGPGAERSKEMPRGSFLLLNTSADSKHTILSPWMRSSSEHCTLAVSVHRHLQPSGRYIAQLLPHNEAAREILLMPTPGKHGWTVLQGRIGRPDNPFRVALEYISSGNRSLSAVDFFALKNCSEGTSPGSKMALQSSFTCWNGTVLQLGQACDFHQDCAQGEDESQMCRKLPVGFYCNFEDGFCGWTQGTLSPHTPQWQVRTLKDARFQDHQDHALLLSTTDVPASESATVTSATFPAPIKSSPCELRMSWLIRGVLRGNVSLVLVENKTGKEQGRMVWHVAAYEGLSLWQWMVLPLLDVSDRFWLQMVAWWGQGSRAIVAFDNISISLDCYLTISGEDKILQNTAPKSRNLFERNPNKELKPGENSPRQTPIFDPTVHWLFTTCGASGPHGPTQAQCNNAYQNSNLSVEVGSEGPLKGIQIWKVPATDTYSISGYGAAGGKGGKNTMMRSHGVSVLGIFNLEKDDMLYILVGQQGEDACPSTNQLIQKVCIGENNVIEEEIRVNRSVHEWAGGGGGGGGATYVFKMKDGVPVPLIIAAGGGGRAYGAKTDTFHPERLENNSSVLGLNGNSGAAGGGGGWNDNTSLLWAGKSLQEGATGGHSCPQAMKKWGWETRGGFGGGGGGCSSGGGGGGYIGGNAASNNDPEMDGEDGVSFISPLGILYTPALKVMEGHGEVNIKHYLNCSHCEVDECHMDPESHKVICFCDHGTVLAEDGVSCIVSPTPEPHLPLSLILSVVTSALVAALVLAFSGIMIVYRRKHQELQAMQMELQSPEYKLSKLRTSTIMTDYNPNYCFAGKTSSISDLKEVPRKNITLIRGLGHGAFGEVYEGQVSGMPNDPSPLQVAVKTLPEVCSEQDELDFLMEALIISKFNHQNIVRCIGVSLQSLPRFILLELMAGGDLKSFLRETRPRPSQPSSLAMLDLLHVARDIACGCQYLEENHFIHRDIAARNCLLTCPGPGRVAKIGDFGMARDIYRASYYRKGGCAMLPVKWMPPEAFMEGIFTSKTDTWSFGVLLWEIFSLGYMPYPSKSNQEVLEFVTSGGRMDPPKNCPGPVYRIMTQCWQHQPEDRPNFAIILERIEYCTQDPDVINTALPIEYGPLVEEEEKVPVRPKDPEGVPPLLVSQQAKREEERSPAAPPPLPTTSSGKAAKKPTAAEISVRVPRGPAVEGGHVNMAFSQSNPPSELHKVHGSRNKPTSLWNPTYGSWFTEKPTKKNNPIAKKEPHDRGNLGLEGSCTVPPNVATGRLPGASLLLEPSSLTANMKEVPLFRLRHFPCGNVNYGYQQQGLPLEAATAPGAGHYEDTILKSKNSMNQPGP",
# ICAM5_HUMAN
"MPGPSPGLRRALLGLWAALGLGLFGLSAVSQEPFWADLQPRVAFVERGGSLWLNCSTNCPRPERGGLETSLRRNGTQRGLRWLARQLVDIREPETQPVCFFRCARRTLQARGLIRTFQRPDRVELMPLPPWQPVGENFTLSCRVPGAGPRASLTLTLLRGAQELIRRSFAGEPPRARGAVLTATVLARREDHGANFSCRAELDLRPHGLGLFENSSAPRELRTFSLSPDAPRLAAPRLLEVGSERPVSCTLDGLFPASEARVYLALGDQNLSPDVTLEGDAFVATATATASAEQEGARQLVCNVTLGGENRETRENVTIYSFPAPLLTLSEPSVSEGQMVTVTCAAGAQALVTLEGVPAAVPGQPAQLQLNATENDDRRSFFCDATLDVDGETLIKNRSAELRVLYAPRLDDSDCPRSWTWPEGPEQTLRCEARGNPEPSVHCARSDGGAVLALGLLGPVTRALSGTYRCKAANDQGEAVKDVTLTVEYAPALDSVGCPERITWLEGTEASLSCVAHGVPPPDVICVRSGELGAVIEGLLRVAREHAGTYRCEATNPRGSAAKNVAVTVEYGPRFEEPSCPSNWTWVEGSGRLFSCEVDGKPQPSVKCVGSGGATEGVLLPLAPPDPSPRAPRIPRVLAPGIYVCNATNRHGSVAKTVVVSAESPPEMDESTCPSHQTWLEGAEASALACAARGRPSPGVRCSREGIPWPEQQRVSREDAGTYHCVATNAHGTDSRTVTVGVEYRPVVAELAASPPGGVRPGGNFTLTCRAEAWPPAQISWRAPPGALNIGLSSNNSTLSVAGAMGSHGGEYECAATNAHGRHARRITVRVAGPWLWVAVGGAAGGAALLAAGAGLAFYVQSTACKKGEYNVQEAESSGEAVCLNGAGGGAGGAAGAEGGPEAAGGAAESPAEGEVFAIQLTSA",
# PACN2_HUMAN
"MSVTYDDSVGVEVSSDSFWEVGNYKRTVKRIDDGHRLCSDLMNCLHERARIEKAYAQQLTEWARRWRQLVEKGPQYGTVEKAWMAFMSEAERVSELHLEVKASLMNDDFEKIKNWQKEAFHKQMMGGFKETKEAEDGFRKAQKPWAKKLKEVEAAKKAHHAACKEEKLAISREANSKADPSLNPEQLKKLQDKIEKCKQDVLKTKEKYEKSLKELDQGTPQYMENMEQVFEQCQQFEEKRLRFFREVLLEVQKHLDLSNVAGYKAIYHDLEQSIRAADAVEDLRWFRANHGPGMAMNWPQFEEWSADLNRTLSRREKKKATDGVTLTGINQTGDQSLPSKPSSTLNVPSNPAQSAQSQSSYNPFEDEDDTGSTVSEKDDTKAKNVSSYEKTQSYPTDWSDDESNNPFSSTDANGDSNPFDDDATSGTEVRVRALYDYEGQEHDELSFKAGDELTKMEDEDEQGWCKGRLDNGQVGLYPANYVEAIQ",
# SNX6_HUMAN
"MMEGLDDGPDFLSEEDRGLKAINVDLQSDAALQVDISDALSERDKVKFTVHTKSSLPNFKQNEFSVVRQHEEFIWLHDSFVENEDYAGYIIPPAPPRPDFDASREKLQKLGEGEGSMTKEEFTKMKQELEAEYLAIFKKTVAMHEVFLCRVAAHPILRRDLNFHVFLEYNQDLSVRGKNKKEKLEDFFKNMVKSADGVIVSGVKDVDDFFEHERTFLLEYHNRVKDASAKSDRMTRSHKSAADDYNRIGSSLYALGTQDSTDICKFFLKVSELFDKTRKIEARVSADEDLKLSDLLKYYLRESQAAKDLLYRRSRSLVDYENANKALDKARAKNKDVLQAETSQQLCCQKFEKISESAKQELIDFKTRRVAAFRKNLVELAELELKHAKGNLQLLQNCLAVLNGDT",
# FZD1_HUMAN
"MAEEEAPKKSRAAGGGASWELCAGALSARLAEEGSGDAGGRRRPPVDPRRLARQLLLLLWLLEAPLLLGVRAQAAGQGPGQGPGPGQQPPPPPQQQQSGQQYNGERGISVPDHGYCQPISIPLCTDIAYNQTIMPNLLGHTNQEDAGLEVHQFYPLVKVQCSAELKFFLCSMYAPVCTVLEQALPPCRSLCERARQGCEALMNKFGFQWPDTLKCEKFPVHGAGELCVGQNTSDKGTPTPSLLPEFWTSNPQHGGGGHRGGFPGGAGASERGKFSCPRALKVPSYLNYHFLGEKDCGAPCEPTKVYGLMYFGPEELRFSRTWIGIWSVLCCASTLFTVLTYLVDMRRFSYPERPIIFLSGCYTAVAVAYIAGFLLEDRVVCNDKFAEDGARTVAQGTKKEGCTILFMMLYFFSMASSIWWVILSLTWFLAAGMKWGHEAIEANSQYFHLAAWAVPAIKTITILALGQVDGDVLSGVCFVGLNNVDALRGFVLAPLFVYLFIGTSFLLAGFVSLFRIRTIMKHDGTKTEKLEKLMVRIGVFSVLYTVPATIVIACYFYEQAFRDQWERSWVAQSCKSYAIPCPHLQAGGGAPPHPPMSPDFTVFMIKYLMTLIVGITSGFWIWSGKTLNSWRKFYTRLTNSKQGETTV",
# BSN_HUMAN
"MGNEVSLEGGAGDGPLPPGGAGPGPGPGPGPGAGKPPSAPAGGGQLPAAGAARSTAVPPVPGPGPGPGPGPGPGSTSRRLDPKEPLGNQRAASPTPKQASATTPGHESPRETRAQGPAGQEADGPRRTLQVDSRTQRSGRSPSVSPDRGSTPTSPYSVPQIAPLPSSTLCPICKTSDLTSTPSQPNFNTCTQCHNKVCNQCGFNPNPHLTQVKEWLCLNCQMQRALGMDMTTAPRSKSQQQLHSPALSPAHSPAKQPLGKPDQERSRGPGGPQPGSRQAETARATSVPGPAQAAAPPEVGRVSPQPPQPTKPSTAEPRPPAGEAPAKSATAVPAGLGATEQTQEGLTGKLFGLGASLLTQASTLMSVQPEADTQGQPAPSKGTPKIVFNDASKEAGPKPLGSGPGPGPAPGAKTEPGARMGPGSGPGALPKTGGTTSPKHGRAEHQAASKAAAKPKTMPKERAICPLCQAELNVGSKSPANYNTCTTCRLQVCNLCGFNPTPHLVEKTEWLCLNCQTKRLLEGSLGEPTPLPPPTSQQPPVGAPHRASGTSPLKQKGPQGLGQPSGPLPAKASPLSTKASPLPSKASPQAKPLRASEPSKTPSSVQEKKTRVPTKAEPMPKPPPETTPTPATPKVKSGVRRAEPATPVVKAVPEAPKGGEAEDLVGKPYSQDASRSPQSLSDTGYSSDGISSSQSEITGVVQQEVEQLDSAGVTGPHPPSPSEIHKVGSSMRPLLQAQGLAPSERSKPLSSGTGEEQKQRPHSLSITPEAFDSDEELEDILEEDEDSAEWRRRREQQDTAESSDDFGSQLRHDYVEDSSEGGLSPLPPQPPARAAELTDEDFMRRQILEMSAEEDNLEEDDTATSGRGLAKHGTQKGGPRPRPEPSQEPAALPKRRLPHNATTGYEELLPEGGSAEATDGSGTLQGGLRRFKTIELNSTGSYGHELDLGQGPDPSLDREPELEMESLTGSPEDRSRGEHSSTLPASTPSYTSGTSPTSLSSLEEDSDSSPSRRQRLEEAKQQRKARHRSHGPLLPTIEDSSEEEELREEEELLREQEKMREVEQQRIRSTARKTRRDKEELRAQRRRERSKTPPSNLSPIEDASPTEELRQAAEMEELHRSSCSEYSPSPSLDSEAEALDGGPSRLYKSGSEYNLPTFMSLYSPTETPSGSSTTPSSGRPLKSAEEAYEEMMRKAELLQRQQGQAAGARGPHGGPSQPTGPRGLGSFEYQDTTDREYGQAAQPAAEGTPASLGAAVYEEILQTSQSIVRMRQASSRDLAFAEDKKKEKQFLNAESAYMDPMKQNGGPLTPGTSPTQLAAPVSFSTPTSSDSSGGRVIPDVRVTQHFAKETQDPLKLHSSPASPSSASKEIGMPFSQGPGTPATTAVAPCPAGLPRGYMTPASPAGSERSPSPSSTAHSYGHSPTTANYGSQTEDLPQAPSGLAAAGRAAREKPLSASDGEGGTPQPSRAYSYFASSSPPLSPSSPSESPTFSPGKMGPRATAEFSTQTPSPAPASDMPRSPGAPTPSPMVAQGTQTPHRPSTPRLVWQESSQEAPFMVITLASDASSQTRMVHASASTSPLCSPTETQPTTHGYSQTTPPSVSQLPPEPPGPPGFPRVPSAGADGPLALYGWGALPAENISLCRISSVPGTSRVEPGPRTPGTAVVDLRTAVKPTPIILTDQGMDLTSLAVEARKYGLALDPIPGRQSTAVQPLVINLNAQEHTFLATATTVSITMASSVFMAQQKQPVVYGDPYQSRLDFGQGGGSPVCLAQVKQVEQAVQTAPYRSGPRGRPREAKFARYNLPNQVAPLARRDVLITQMGTAQSIGLKPGPVPEPGAEPHRATPAELRSHALPGARKPHTVVVQMGEGTAGTVTTLLPEEPAGALDLTGMRPESQLACCDMVYKLPFGSSCTGTFHPAPSVPEKSMADAAPPGQSSSPFYGPRDPEPPEPPTYRAQGVVGPGPHEEQRPYPQGLPGRLYSSMSDTNLAEAGLNYHAQRIGQLFQGPGRDSAMDLSSLKHSYSLGFADGRYLGQGLQYGSVTDLRHPTDLLAHPLPMRRYSSVSNIYSDHRYGPRGDAVGFQEASLAQYSATTAREISRMCAALNSMDQYGGRHGSGGGGPDLVQYQPQHGPGLSAPQSLVPLRPGLLGNPTFPEGHPSPGNLAQYGPAAGQGTAVRQLLPSTATVRAADGMIYSTINTPIAATLPITTQPASVLRPMVRGGMYRPYASGGITAVPLTSLTRVPMIAPRVPLGPTGLYRYPAPSRFPIASSVPPAEGPVYLGKPAAAKAPGAGGPSRPEMPVGAAREEPLPTTTPAAIKEAAGAPAPAPLAGQKPPADAAPGGGSGALSRPGFEKEEASQEERQRKQQEQLLQLERERVELEKLRQLRLQEELERERVELQRHREEEQLLVQRELQELQTIKHHVLQQQQEERQAQFALQREQLAQQRLQLEQIQQLQQQLQQQLEEQKQRQKAPFPAACEAPGRGPPLAAAELAQNGQYWPPLTHAAFIAMAGPEGLGQPREPVLHRGLPSSASDMSLQTEEQWEASRSGIKKRHSMPRLRDACELESGTEPCVVRRIADSSVQTDDEDGESRYLLSRRRRARRSADCSVQTDDEDSAEWEQPVRRRRSRLPRHSDSGSDSKHDATASSSSAAATVRAMSSVGIQTISDCSVQTEPDQLPRVSPAIHITAATDPKVEIVRYISAPEKTGRGESLACQTEPDGQAQGVAGPQLVGPTAISPYLPGIQIVTPGPLGRFEKKKPDPLEIGYQAHLPPESLSQLVSRQPPKSPQVLYSPVSPLSPHRLLDTSFASSERLNKAHVSPQKHFTADSALRQQTLPRPMKTLQRSLSDPKPLSPTAEESAKERFSLYQHQGGLGSQVSALPPNSLVRKVKRTLPSPPPEEAHLPLAGQASPQLYAASLLQRGLTGPTTVPATKASLLRELDRDLRLVEHESTKLRKKQAELDEEEKEIDAKLKYLELGITQRKESLAKDRGGRDYPPLRGLGEHRDYLSDSELNQLRLQGCTTPAGQFVDFPATAAAPATPSGPTAFQQPRFQPPAPQYSAGSGGPTQNGFPAHQAPTYPGPSTYPAPAFPPGASYPAEPGLPNQQAFRPTGHYAGQTPMPTTQSTLFPVPADSRAPLQKPRQTSLADLEQKVPTNYEVIASPVVPMSSAPSETSYSGPAVSSGYEQGKVPEVPRAGDRGSVSQSPAPTYPSDSHYTSLEQNVPRNYVMIDDISELTKDSTSTAPDSQRLEPLGPGSSGRPGKEPGEPGVLDGPTLPCCYARGEEESEEDSYDPRGKGGHLRSMESNGRPASTHYYGDSDYRHGARVEKYGPGPMGPKHPSKSLAPAAISSKRSKHRKQGMEQKISKFSPIEEAKDVESDLASYPPPAVSSSLVSRGRKFQDEITYGLKKNVYEQQKYYGMSSRDAVEDDRIYGGSSRSRAPSAYSGEKLSSHDFSGWGKGYEREREAVERLQKAGPKPSSLSMAHSRVRPPMRSQASEEESPVSPLGRPRPAGGPLPPGGDTCPQFCSSHSMPDVQEHVKDGPRAHAYKREEGYILDDSHCVVSDSEAYHLGQEETDWFDKPRDARSDRFRHHGGHAVSSSSQKRGPARHSYHDYDEPPEEGLWPHDEGGPGRHASAKEHRHGDHGRHSGRHTGEEPGRRAAKPHARDLGRHEARPHSQPSSAPAMPKKGQPGYPSSAEYSQPSRASSAYHHASDSKKGSRQAHSGPAALQSKAEPQAQPQLQGRQAAPGPQQSQSPSSRQIPSGAASRQPQTQQQQQGLGLQPPQQALTQARLQQQSQPTTRGSAPAASQPAGKPQPGPSTATGPQPAGPPRAEQTNGSKGTAKAPQQGRAPQAQPAPGPGPAGVKAGARPGGTPGAPAGQPGADGESVFSKILPGGAAEQAGKLTEAVSAFGKKFSSFW",
# IQEC3_HUMAN
"MESLLENPVRAVLYLKELTAIVQNQQSLIHTQRERIDELERRLDELSAENRSLWEHQQLLQAQPPPGLVPPSSAPLPAAPATAPAAAARAQEPLQDQGQRSAAAPHPAPDRPPRQHHGQLLEQPQRGPGSRAHTPQSPHKHLGTQGAVTDKEKERPPSCCAAAGALLQHKSPSALGKGVLSRRPENETVLHQFCCPAADACSDLASQSDGSCTQAGGGMEDSVVAAAAVAAGRPSAHAPKAQAQELQEEEERPGAGAASPRAGPQHKASPGRQQPALATALCPHAPAASDYELSLDLKNKQIEMLEHKYGGHLVSRRAACTIQTAFRQYQLSKNFEKIRNSLLESRLPRRISLRKVRSPTAESLAAEKALMEGYGLVGLPLVRSPSLPPTFAGTLTELEDSFTEQVQSLAKSIDDALSTWSLKTMCSLRESGAYQLHQALQAAAGPPGLEAEGRAPESAGPGPGDDAAETPGLPPAHSGTLMMAFRDVTVQIANQNISVSSSTALSVANCLGAQTVQAPAEPAAGKAEQGETSGREAPEAPAVGREDASAEDSCAEAAASGAADGATAPKTEEEEEEEETAEVGRGAEAEAGDLEQLSSSSTSTKSAKSGSEASASASKDALQAMILSLPRYHCENPASCKSPTLSTDTLRKRLYRIGLNLFNINPDKGIQFLISRGFIPDTPIGVAHFLLQRKGLSRQMIGEFLGNSKKQFNRDVLDCVVDEMDFSSMELDEALRKFQAHIRVQGEAQKVERLIEAFSQRYCMCNPEVVQQFHNPDTIFILAFAIILLNTDMYSPNIKPDRKMMLEDFIRNLRGVDDGADIPRELVVGIYERIQQKELKSNEDHVTYVTKVEKSIVGMKTVLSVPHRRLVCCSRLFEVTDVNKLQKQAAHQREVFLFNDLLVILKLCPKKKSSSTYTFCKSVGLLGMQFQLFENEYYSHGITLVTPLSGSEKKQVLHFCALGSDEMQKFVEDLKESIAEVTELEQIRIEWELEKQQGTKTLSFKPCGAQGDPQSKQGSPTAKREAALRERPAESTVEVSIHNRLQTSQHNSGLGAERGAPVPPPDLQPSPPRQQTPPLPPPPPTPPGTLVQCQQIVKVIVLDKPCLARMEPLLSQALSCYTSSSSDSCGSTPLGGPGSPVKVTHQPPLPPPPPPYNHPHQFCPPGSLLHGHRYSSGSRSLV",
# AGAP1_HUMAN
"MNYQQQLANSAAIRAEIQRFESVHPNIYSIYELLERVEEPVLQNQIREHVIAIEDAFVNSQEWTLSRSVPELKVGIVGNLASGKSALVHRYLTGTYVQEESPEGGRFKKEIVVDGQSYLLLIRDEGGPPEAQFAMWVDAVIFVFSLEDEISFQTVYHYYSRMANYRNTSEIPLVLVGTQDAISSANPRVIDDARARKLSNDLKRCTYYETCATYGLNVERVFQDVAQKIVATRKKQQLSIGPCKSLPNSPSHSSVCSAQVSAVHISQTSNGGGSLSDYSSSVPSTPSTSQKELRIDVPPTANTPTPVRKQSKRRSNLFTSRKGSDPDKEKKGLESRADSIGSGRAIPIKQGMLLKRSGKSLNKEWKKKYVTLCDNGVLTYHPSLHDYMQNVHGKEIDLLRTTVKVPGKRPPRATSACAPISSPKTNGLSKDMSSLHISPNSGNVTSASGSQMASGISLVSFNSRPDGMHQRSYSVSSADQWSEATVIANSAISSDTGLGDSVCSSPSISSTTSPKLDPPPSPHANRKKHRRKKSTSNFKADGLSGTAEEQEENFEFIIVSLTGQTWHFEATTYEERDAWVQAIESQILASLQSCESSKNKSRLTSQSEAMALQSIRNMRGNSHCVDCETQNPNWASLNLGALMCIECSGIHRNLGTHLSRVRSLDLDDWPVELIKVMSSIGNELANSVWEESSQGRTKPSVDSTREEKERWIRAKYEQKLFLAPLPCTELSLGQHLLRATADEDLRTAILLLAHGSRDEVNETCGEGDGRTALHLACRKGNVVLAQLLIWYGVDVTARDAHGNTALAYARQASSQECIDVLLQYGCPDERFVLMATPNLSRRNNNRNNSSGRVPTII",
# NAC2_HUMAN
"MAPLALVGVTLLLAAPPCSGAATPTPSLPPPPANDSDTSTGGCQGSYRCQPGVLLPVWEPDDPSLGDKAARAVVYFVAMVYMFLGVSIIADRFMAAIEVITSKEKEITITKANGETSVGTVRIWNETVSNLTLMALGSSAPEILLSVIEVCGHNFQAGELGPGTIVGSAAFNMFVVIAVCIYVIPAGESRKIKHLRVFFVTASWSIFAYVWLYLILAVFSPGVVQVWEALLTLVFFPVCVVFAWMADKRLLFYKYVYKRYRTDPRSGIIIGAEGDPPKSIELDGTFVGAEAPGELGGLGPGPAEARELDASRREVIQILKDLKQKHPDKDLEQLVGIANYYALLHQQKSRAFYRIQATRLMTGAGNVLRRHAADASRRAAPAEGAGEDEDDGASRIFFEPSLYHCLENCGSVLLSVTCQGGEGNSTFYVDYRTEDGSAKAGSDYEYSEGTLVFKPGETQKELRIGIIDDDIFEEDEHFFVRLLNLRVGDAQGMFEPDGGGRPKGRLVAPLLATVTILDDDHAGIFSFQDRLLHVSECMGTVDVRVVRSSGARGTVRLPYRTVDGTARGGGVHYEDACGELEFGDDETMKTLQVKIVDDEEYEKKDNFFIELGQPQWLKRGISALLLNQGDGDRKLTAEEEEARRIAEMGKPVLGENCRLEVIIEESYDFKNTVDKLIKKTNLALVIGTHSWREQFLEAITVSAGDEEEEEDGSREERLPSCFDYVMHFLTVFWKVLFACVPPTEYCHGWACFGVSILVIGLLTALIGDLASHFGCTVGLKDSVNAVVFVALGTSIPDTFASKVAALQDQCADASIGNVTGSNAVNVFLGLGVAWSVAAVYWAVQGRPFEVRTGTLAFSVTLFTVFAFVGIAVLLYRRRPHIGGELGGPRGPKLATTALFLGLWLLYILFASLEAYCHIRGF",
# EXOC7_HUMAN
"MIPPQEASARRREIEDKLKQEEETLSFIRDSLEKSDQLTKNMVSILSSFESRLMKLENSIIPVHKQTENLQRLQENVEKTLSCLDHVISYYHVASDTEKIIREGPTGRLEEYLGSMAKIQKAVEYFQDNSPDSPELNKVKLLFERGKEALESEFRSLMTRHSKVVSPVLILDLISGDDDLEAQEDVTLEHLPESVLQDVIRISRWLVEYGRNQDFMNVYYQIRSSQLDRSIKGLKEHFHKSSSSSGVPYSPAIPNKRKDTPTKKPVKRPGTIRKAQNLLKQYSQHGLDGKKGGSNLIPLEGLLPCTPRGGLPGPWINAACVCAADISPGHEHDFRVKHLSEALNDKHGPLAGRDDMLDVETDAYIHCVSAFVKLAQSEYQLLADIIPEHHQKKTFDSLIQDALDGLMLEGENIVSAARKAIVRHDFSTVLTVFPILRHLKQTKPEFDQVLQGTAASTKNKLPGLITSMETIGAKALEDFADNIKNDPDKEYNMPKDGTVHELTSNAILFLQQLLDFQETAGAMLASQETSSSATSYSSEFSKRLLSTYICKVLGNLQLNLLSKSKVYEDPALSAIFLHNNYNYILKSLEKSELIQLVAVTQKTAERSYREHIEQQIQTYQRSWLKVTDYIAEKNLPVFQPGVKLRDKERQIIKERFKGFNDGLEELCKIQKAWAIPDTEQRDRIRQAQKTIVKETYGAFLQKFGSVPFTKNPEKYIKYGVEQVGDMIDRLFDTSA",
# SORC3_HUMAN
"MEAARTERPAGRPGAPLVRTGLLLLSTWVLAGAEITWDATGGPGRPAAPASRPPALSPLSPRAVASQWPEELASARRAAVLGRRAGPELLPQQGGGRGGEMQVEAGGTSPAGERRGRGIPAPAKLGGARRSRRAQPPITQERGDAWATAPADGSRGSRPLAKGSREEVKAPRAGGSAAEDLRLPSTSFALTGDSAHNQAMVHWSGHNSSVILILTKLYDFNLGSVTESSLWRSTDYGTTYEKLNDKVGLKTVLSYLYVNPTNKRKIMLLSDPEMESSILISSDEGATYQKYRLTFYIQSLLFHPKQEDWVLAYSLDQKLYSSMDFGRRWQLMHERITPNRFYWSVAGLDKEADLVHMEVRTTDGYAHYLTCRIQECAETTRSGPFARSIDISSLVVQDEYIFIQVTTSGRASYYVSYRREAFAQIKLPKYSLPKDMHIISTDENQVFAAVQEWNQNDTYNLYISDTRGIYFTLAMENIKSSRGLMGNIIIELYEVAGIKGIFLANKKVDDQVKTYITYNKGRDWRLLQAPDVDLRGSPVHCLLPFCSLHLHLQLSENPYSSGRISSKETAPGLVVATGNIGPELSYTDIGVFISSDGGNTWRQIFDEEYNVWFLDWGGALVAMKHTPLPVRHLWVSFDEGHSWDKYGFTSVPLFVDGALVEAGMETHIMTVFGHFSLRSEWQLVKVDYKSIFSRHCTKEDYQTWHLLNQGEPCVMGERKIFKKRKPGAQCALGRDHSGSVVSEPCVCANWDFECDYGYERHGESQCVPAFWYNPASPSKDCSLGQSYLNSTGYRRIVSNNCTDGLREKYTAKAQMCPGKAPRGLHVVTTDGRLVAEQGHNATFIILMEEGDLQRTNIQLDFGDGIAVSYANFSPIEDGIKHVYKSAGIFQVTAYAENNLGSDTAVLFLHVVCPVEHVHLRVPFVAIRNKEVNISAVVWPSQLGTLTYFWWFGNSTKPLITLDSSISFTFLAEGTDTITVQVAAGNALIQDTKEIAVHEYFQSQLLSFSPNLDYHNPDIPEWRKDIGNVIKRALVKVTSVPEDQILIAVFPGLPTSAELFILPPKNLTERRKGNEGDLEQIVETLFNALNQNLVQFELKPGVQVIVYVTQLTLAPLVDSSAGHSSSAMLMLLSVVFVGLAVFLIYKFKRKIPWINIYAQVQHDKEQEMIGSVSQSENAPKITLSDFTEPEELLDKELDTRVIGGIATIANSESTKEIPNCTSV",
# TBD2B_HUMAN
"MPGAGARAEEGGGGGEGAAQGAAAEPGAGPAREPARLCGYLQKLSGKGPLRGYRSRWFVFDARRCYLYYFKSPQDALPLGHLDIADACFSYQGPDEAAEPGTEPPAHFQVHSAGAVTVLKAPNRQLMTYWLQELQQKRWEYCNSLDMVKWDSRTSPTPGDFPKGLVARDNTDLIYPHPNASAEKARNVLAVETVPGELVGEQAANQPAPGHPNSINFYSLKQWGNELKNSMSSFRPGRGHNDSRRTVFYTNEEWELLDPTPKDLEESIVQEEKKKLTPEGNKGVTGSGFPFDFGRNPYKGKRPLKDIIGSYKNRHSSGDPSSEGTSGSGSVSIRKPASEMQLQVQSQQEELEQLKKDLSSQKELVRLLQQTVRSSQYDKYFTSSRLCEGVPKDTLELLHQKDDQILGLTSQLERFSLEKESLQQEVRTLKSKVGELNEQLGMLMETIQAKDEVIIKLSEGEGNGPPPTVAPSSPSVVPVARDQLELDRLKDNLQGYKTQNKFLNKEILELSALRRNAERRERDLMAKYSSLEAKLCQIESKYLILLQEMKTPVCSEDQGPTREVIAQLLEDALQVESQEQPEQAFVKPHLVSEYDIYGFRTVPEDDEEEKLVAKVRALDLKTLYLTENQEVSTGVKWENYFASTVNREMMCSPELKNLIRAGIPHEHRSKVWKWCVDRHTRKFKDNTEPGHFQTLLQKALEKQNPASKQIELDLLRTLPNNKHYSCPTSEGIQKLRNVLLAFSWRNPDIGYCQGLNRLVAVALLYLEQEDAFWCLVTIVEVFMPRDYYTKTLLGSQVDQRVFRDLMSEKLPRLHGHFEQYKVDYTLITFNWFLVVFVDSVVSDILFKIWDSFLYEGPKVIFRFALALFKYKEEEILKLQDSMSIFKYLRYFTRTILDARKLISISFGDLNPFPLRQIRNRRAYHLEKVRLELTELEAIREDFLRERDTSPDKGELVSDEEEDT",
# SMAG1_HUMAN
"MMFRDQVGVLAGWFKGWNECEQTVALLSLLKRVSQTQARFLQLCLEHSLADCAELHVLEREANSPGIINQWQQESKDKVISLLLTHLPLLKPGNLDAKVEYMKLLPKILAHSIEHNQHIEESRQLLSYALIHPATSLEDRSALAMWLNHLEDRTSTSFGGQNRGRSDSVDYGQTHYYHQRQNSDDKLNGWQNSRDSGICINASNWQDKSMGCENGHVPLYSSSSVPTTINTIGTSTSTILSGQAHHSPLKRSVSLTPPMNVPNQPLGHGWMSHEDLRARGPQCLPSDHAPLSPQSSVASSGSGGSEHLEDQTTARNTFQEEGSGMKDVPAWLKSLRLHKYAALFSQMTYEEMMALTECQLEAQNVTKGARHKIVISIQKLKERQNLLKSLERDIIEGGSLRIPLQELHQMILTPIKAYSSPSTTPEARRREPQAPRQPSLMGPESQSPDCKDGAAATGATATPSAGASGGLQPHQLSSCDGELAVAPLPEGDLPGQFTRVMGKVCTQLLVSRPDEENISSYLQLIDKCLIHEAFTETQKKRLLSWKQQVQKLFRSFPRKTLLDISGYRQQRNRGFGQSNSLPTAGSVGGGMGRRNPRQYQIPSRNVPSARLGLLGTSGFVSSNQRNTTATPTIMKQGRQNLWFANPGGSNSMPSRTHSSVQRTRSLPVHTSPQNMLMFQQPEFQLPVTEPDINNRLESLCLSMTEHALGDGVDRTSTI",
# PHF24_HUMAN
"MGVLMSKRQTVEQVQKVSLAVSAFKDGLRDRPSIRRTGELPGSRRGTVEGSVQEVQEEKEAEAGTSVVQEESSAGRAAWERLRDGRGVEPEEFDRTSRFTPPAFIRPTRKLDDDKPPEICLEPREPVVNDEMCDVCEVWTAESLFPCRVCTRVFHDGCLRRMGYIQGDSAAEVTEMAHTETGWSCHYCDNINLLLTEEEMYSLTETFQRCKVIPDCSLTLEDFLRYRHQAAKRGDRDRALSEEQEEQAARQFAALDPEHRGHIEWPDFLSHESLLLLQQLRPQNSLLRLLTVKERERARAAFLARGSGSTVSEAECRRAQHSWFCKRFPEAPSCSVSISHVGPIADSSPASSSSKSQDKTLLPTEQESRFVDWPTFLQENVLYILAARPNSAAIHLKPPG",
# UN13A_HUMAN
"MSLLCVGVKKAKFDGAQEKFNTYVTLKVQNVKSTTIAVRGSQPSWEQDFMFEINRLDLGLTVEVWNKGLIWDTMVGTVWIPLRTIRQSNEEGPGEWLTLDSQVIMADSEICGTKDPTFHRILLDTRFELPLDIPEEEARYWAKKLEQLNAMRDQDEYSFQDEQDKPLPVPSNQCCNWNYFGWGEQHNDDPDSAVDDRDSDYRSETSNSIPPPYYTTSQPNASVHQYSVRPPPLGSRESYSDSMHSYEEFSEPQALSPTGSSRYASSGELSQGSSQLSEDFDPDEHSLQGSDMEDERDRDSYHSCHSSVSYHKDSPRWDQDEEELEEDLEDFLEEEELPEDEEELEEEEEEVPDDLGSYAQREDVAVAEPKDFKRISLPPAAPGKEDKAPVAPTEAPDMAKVAPKPATPDKVPAAEQIPEAEPPKDEESFRPREDEEGQEGQDSMSRAKANWLRAFNKVRMQLQEARGEGEMSKSLWFKGGPGGGLIIIDSMPDIRKRKPIPLVSDLAMSLVQSRKAGITSALASSTLNNEELKNHVYKKTLQALIYPISCTTPHNFEVWTATTPTYCYECEGLLWGIARQGMRCTECGVKCHEKCQDLLNADCLQRAAEKSSKHGAEDRTQNIIMVLKDRMKIRERNKPEIFELIQEIFAVTKTAHTQQMKAVKQSVLDGTSKWSAKISITVVCAQGLQAKDKTGSSDPYVTVQVGKTKKRTKTIYGNLNPVWEENFHFECHNSSDRIKVRVWDEDDDIKSRVKQRFKRESDDFLGQTIIEVRTLSGEMDVWYNLDKRTDKSAVSGAIRLHISVEIKGEEKVAPYHVQYTCLHENLFHFVTDVQNNGVVKIPDAKGDDAWKVYYDETAQEIVDEFAMRYGVESIYQAMTHFACLSSKYMCPGVPAVMSTLLANINAYYAHTTASTNVSASDRFAASNFGKERFVKLLDQLHNSLRIDLSMYRNNFPASSPERLQDLKSTVDLLTSITFFRMKVQELQSPPRASQVVKDCVKACLNSTYEYIFNNCHELYSREYQTDPAKKGEVLPEEQGPSIKNLDFWSKLITLIVSIIEEDKNSYTPCLNQFPQELNVGKISAEVMWNLFAQDMKYAMEEHDKHRLCKSADYMNLHFKVKWLYNEYVTELPAFKDRVPEYPAWFEPFVIQWLDENEEVSRDFLHGALERDKKDGFQQTSEHALFSCSVVDVFSQLNQSFEIIKKLECPDPQIVGHYMRRFAKTISNVLLQYADIISKDFASYCSKEKEKVPCILMNNTQQLRVQLEKMFEAMGGKELDAEASDILKELQVKLNNVLDELSRVFATSFQPHIEECVKQMGDILSQVKGTGNVPASACSSVAQDADNVLQPIMDLLDSNLTLFAKICEKTVLKRVLKELWKLVMNTMEKTIVLPPLTDQTMIGNLLRKHGKGLEKGRVKLPSHSDGTQMIFNAAKELGQLSKLKDHMVREEAKSLTPKQCAVVELALDTIKQYFHAGGVGLKKTFLEKSPDLQSLRYALSLYTQATDLLIKTFVQTQSAQGLGVEDPVGEVSVHVELFTHPGTGEHKVTVKVVAANDLKWQTSGIFRPFIEVNIIGPQLSDKKRKFATKSKNNSWAPKYNESFQFTLSADAGPECYELQVCVKDYCFAREDRTVGLAVLQLRELAQRGSAACWLPLGRRIHMDDTGLTVLRILSQRSNDEVAKEFVKLKSDTRSAEEGGAAPAP",
# TUTLB_HUMAN
"MIWYVATFIASVIGTRGLAAEGAHGLREEPEFVTARAGESVVLRCDVIHPVTGQPPPYVVEWFKFGVPIPIFIKFGYYPPHVDPEYAGRASLHDKASLRLEQVRSEDQGWYECKVLMLDQQYDTFHNGSWVHLTINAPPTFTETPPQYIEAKEGGSITMTCTAFGNPKPIVTWLKEGTLLGASGKYQVSDGSLTVTSVSREDRGAYTCRAYSIQGEAVHTTHLLVQGPPFIVSPPENITVNISQDALLTCRAEAYPGNLTYTWYWQDENVYFQNDLKLRVRILIDGTLIIFRVKPEDSGKYTCVPSNSLGRSPSASAYLTVQYPARVLNMPPVIYVPVGIHGYIRCPVDAEPPATVVKWNKDGRPLQVEKNLGWTLMEDGSIRIEEATEEALGTYTCVPYNTLGTMGQSAPARLVLKDPPYFTVLPGWEYRQEAGRELLIPCAAAGDPFPVITWRKVGKPSRSKHSALPSGSLQFRALSKEDHGEWECVATNVVTSITASTHLTVIGTSPHAPGSVRVQVSMTTANVSWEPGYDGGYEQTFSVWMKRAQFGPHDWLSLPVPPGPSWLLVDTLEPETAYQFSVLAQNKLGTSAFSEVVTVNTLAFPITTPEPLVLVTPPRCLIANRTQQGVLLSWLPPANHSFPIDRYIMEFRVAERWELLDDGIPGTEGEFFAKDLSQDTWYEFRVLAVMQDLISEPSNIAGVSSTDIFPQPDLTEDGLARPVLAGIVATICFLAAAILFSTLAACFVNKQRKRKLKRKKDPPLSITHCRKSLESPLSSGKVSPESIRTLRAPSESSDDQGQPAAKRMLSPTREKELSLYKKTKRAISSKKYSVAKAEAEAEATTPIELISRGPDGRFVMDPAEMEPSLKSRRIEGFPFAEETDMYPEFRQSDEENEDPLVPTSVAALKSQLTPLSSSQESYLPPPAYSPRFQPRGLEGPGGLEGRLQATGQARPPAPRPFHHGQYYGYLSSSSPGEVEPPPFYVPEVGSPLSSVMSSPPLPTEGPFGHPTIPEENGENASNSTLPLTQTPTGGRSPEPWGRPEFPFGGLETPAMMFPHQLPPCDVPESLQPKAGLPRGLPPTSLQVPAAYPGILSLEAPKGWAGKSPGRGPVPAPPAAKWQDRPMQPLVSQGQLRHTSQGMGIPVLPYPEPAEPGAHGGPSTFGLDTRWYEPQPRPRPSPRQARRAEPSLHQVVLQPSRLSPLTQSPLSSRTGSPELAARARPRPGLLQQAEMSEITLQPPAAVSFSRKSTPSTGSPSQSSRSGSPSYRPAMGFTTLATGYPSPPPGPAPAGPGDSLDVFGQTPSPRRTGEELLRPETPPPTLPTSGKLQRDRPAPATSPPERALSKL",
# SHAN2_HUMAN
"MPRSPTSSEDEMAQSFSDYSVGSESDSSKEETIYDTIRATAEKPGGARTEESQGNTLVIRVVIHDLQQTKCIRFNPDATVWVAKQRILCTLTQSLKDVLNYGLFQPASNGRDGKFLDEERLLREYPQPVGEGVPSLEFRYKKRVYKQASLDEKQLAKLHTKTNLKKCMDHIQHRLVEKITKMLDRGLDPNFHDPETGETPLTLAAQLDDSVEVIKALKNGGAHLDFRAKDGMTALHKAARARNQVALKTLLELGASPDYKDSYGLTPLYHTAIVGGDPYCCELLLHEHATVCCKDENGWHEIHQACRYGHVQHLEHLLFYGADMSAQNASGNTALHICALYNQDSCARVLLFRGGNKELKNYNSQTPFQVAIIAGNFELAEYIKNHKETDIVPFREAPAYSNRRRRPPNTLAAPRVLLRSNSDNNLNASAPDWAVCSTATSHRSLSPQLLQQMPSKPEGAAKTIGSYVPGPRSRSPSLNRLGGAGEDGKRPQPLWHVGSPFALGANKDSLSAFEYPGPKRKLYSAVPGRLFVAVKPYQPQVDGEIPLHRGDRVKVLSIGEGGFWEGSARGHIGWFPAECVEEVQCKPRDSQAETRADRSKKLFRHYTVGSYDSFDTSSDCIIEEKTVVLQKKDNEGFGFVLRGAKADTPIEEFTPTPAFPALQYLESVDEGGVAWQAGLRTGDFLIEVNNENVVKVGHRQVVNMIRQGGNHLVLKVVTVTRNLDPDDTARKKAPPPPKRAPTTALTLRSKSMTSELEELVDKASVRKKKDKPEEIVPASKPSRAAENMAVEPRVATIKQRPSSRCFPAGSDMNSVYERQGIAVMTPTVPGSPKAPFLGIPRGTMRRQKSIDSRIFLSGITEEERQFLAPPMLKFTRSLSMPDTSEDIPPPPQSVPPSPPPPSPTTYNCPKSPTPRVYGTIKPAFNQNSAAKVSPATRSDTVATMMREKGMYFRRELDRYSLDSEDLYSRNAGPQANFRNKRGQMPENPYSEVGKIASKAVYVPAKPARRKGMLVKQSNVEDSPEKTCSIPIPTIIVKEPSTSSSGKSSQGSSMEIDPQAPEPPSQLRPDESLTVSSPFAAAIAGAVRDREKRLEARRNSPAFLSTDLGDEDVGLGPPAPRTRPSMFPEEGDFADEDSAEQLSSPMPSATPREPENHFVGGAEASAPGEAGRPLNSTSKAQGPESSPAVPSASSGTAGPGNYVHPLTGRLLDPSSPLALALSARDRAMKESQQGPKGEAPKADLNKPLYIDTKMRPSLDAGFPTVTRQNTRGPLRRQETENKYETDLGRDRKGDDKKNMLIDIMDTSQQKSAGLLMVHTVDATKLDNALQEEDEKAEVEMKPDSSPSEVPEGVSETEGALQISAAPEPTTVPGRTIVAVGSMEEAVILPFRIPPPPLASVDLDEDFIFTEPLPPPLEFANSFDIPDDRAASVPALSDLVKQKKSDTPQSPSLNSSQPTNSADSKKPASLSNCLPASFLPPPESFDAVADSGIEEVDSRSSSDHHLETTSTISTVSSISTLSSEGGENVDTCTVYADGQAFMVDKPPVPPKPKMKPIIHKSNALYQDALVEEDVDSFVIPPPAPPPPPGSAQPGMAKVLQPRTSKLWGDVTEIKSPILSGPKANVISELNSILQQMNREKLAKPGEGLDSPMGAKSASLAPRSPEIMSTISGTRSTTVTFTVRPGTSQPITLQSRPPDYESRTSGTRRAPSPVVSPTEMNKETLPAPLSAATASPSPALSDVFSLPSQPPSGDLFGLNPAGRSRSPSPSILQQPISNKPFTTKPVHLWTKPDVADWLESLNLGEHKEAFMDNEIDGSHLPNLQKEDLIDLGVTRVGHRMNIERALKQLLDR",
# DICER_HUMAN
"MKSPALQPLSMAGLQLMTPASSPMGPFFGLPWQQEAIHDNIYTPRKYQVELLEAALDHNTIVCLNTGSGKTFIAVLLTKELSYQIRGDFSRNGKRTVFLVNSANQVAQQVSAVRTHSDLKVGEYSNLEVNASWTKERWNQEFTKHQVLIMTCYVALNVLKNGYLSLSDINLLVFDECHLAILDHPYREIMKLCENCPSCPRILGLTASILNGKCDPEELEEKIQKLEKILKSNAETATDLVVLDRYTSQPCEIVVDCGPFTDRSGLYERLLMELEEALNFINDCNISVHSKERDSTLISKQILSDCRAVLVVLGPWCADKVAGMMVRELQKYIKHEQEELHRKFLLFTDTFLRKIHALCEEHFSPASLDLKFVTPKVIKLLEILRKYKPYERQQFESVEWYNNRNQDNYVSWSDSEDDDEDEEIEEKEKPETNFPSPFTNILCGIIFVERRYTAVVLNRLIKEAGKQDPELAYISSNFITGHGIGKNQPRNKQMEAEFRKQEEVLRKFRAHETNLLIATSIVEEGVDIPKCNLVVRFDLPTEYRSYVQSKGRARAPISNYIMLADTDKIKSFEEDLKTYKAIEKILRNKCSKSVDTGETDIDPVMDDDDVFPPYVLRPDDGGPRVTINTAIGHINRYCARLPSDPFTHLAPKCRTRELPDGTFYSTLYLPINSPLRASIVGPPMSCVRLAERVVALICCEKLHKIGELDDHLMPVGKETVKYEEELDLHDEEETSVPGRPGSTKRRQCYPKAIPECLRDSYPRPDQPCYLYVIGMVLTTPLPDELNFRRRKLYPPEDTTRCFGILTAKPIPQIPHFPVYTRSGEVTISIELKKSGFMLSLQMLELITRLHQYIFSHILRLEKPALEFKPTDADSAYCVLPLNVVNDSSTLDIDFKFMEDIEKSEARIGIPSTKYTKETPFVFKLEDYQDAVIIPRYRNFDQPHRFYVADVYTDLTPLSKFPSPEYETFAEYYKTKYNLDLTNLNQPLLDVDHTSSRLNLLTPRHLNQKGKALPLSSAEKRKAKWESLQNKQILVPELCAIHPIPASLWRKAVCLPSILYRLHCLLTAEELRAQTASDAGVGVRSLPADFRYPNLDFGWKKSIDSKSFISISNSSSAENDNYCKHSTIVPENAAHQGANRTSSLENHDQMSVNCRTLLSESPGKLHVEVSADLTAINGLSYNQNLANGSYDLANRDFCQGNQLNYYKQEIPVQPTTSYSIQNLYSYENQPQPSDECTLLSNKYLDGNANKSTSDGSPVMAVMPGTTDTIQVLKGRMDSEQSPSIGYSSRTLGPNPGLILQALTLSNASDGFNLERLEMLGDSFLKHAITTYLFCTYPDAHEGRLSYMRSKKVSNCNLYRLGKKKGLPSRMVVSIFDPPVNWLPPGYVVNQDKSNTDKWEKDEMTKDCMLANGKLDEDYEEEDEEEESLMWRAPKEEADYEDDFLEYDQEHIRFIDNMLMGSGAFVKKISLSPFSTTDSAYEWKMPKKSSLGSMPFSSDFEDFDYSSWDAMCYLDPSKAVEEDDFVVGFWNPSEENCGVDTGKQSISYDLHTEQCIADKSIADCVEALLGCYLTSCGERAAQLFLCSLGLKVLPVIKRTDREKALCPTRENFNSQQKNLSVSCAAASVASSRSSVLKDSEYGCLKIPPRCMFDHPDADKTLNHLISGFENFEKKINYRFKNKAYLLQAFTHASYHYNTITDCYQRLEFLGDAILDYLITKHLYEDPRQHSPGVLTDLRSALVNNTIFASLAVKYDYHKYFKAVSPELFHVIDDFVQFQLEKNEMQGMDSELRRSEEDEEKEEDIEVPKAMGDIFESLAGAIYMDSGMSLETVWQVYYPMMRPLIEKFSANVPRSPVRELLEMEPETAKFSPAERTYDGKVRVTVEVVGKGKFKGVGRSYRIAKSAAARRALRSLKANQPQVPNS",
# WASF3_HUMAN
"MPLVKRNIEPRHLCRGALPEGITSELECVTNSTLAAIIRQLSSLSKHAEDIFGELFNEANNFYIRANSLQDRIDRLAVKVTQLDSTVEEVSLQDINMKKAFKSSTVQDQQVVSKNSIPNPVADIYNQSDKPPPLNILTPYRDDKKDGLKFYTDPSYFFDLWKEKMLQDTEDKRKEKRRQKEQKRIDGTTREVKKVRKARNRRQEWNMMAYDKELRPDNRLSQSVYHGASSEGSLSPDTRSHASDVTDYSYPATPNHSLHPQPVTPSYAAGDVPPHGPASQAAEHEYRPPSASARHMALNRPQQPPPPPPPQAPEGSQASAPMAPADYGMLPAQIIEYYNPSGPPPPPPPPVIPSAQTAFVSPLQMPMQPPFPASASSTHAAPPHPPSTGLLVTAPPPPGPPPPPPGPPGPGSSLSSSPMHGPPVAEAKRQEPAQPPISDARSDLLAAIRMGIQLKKVQEQREQEAKREPVGNDVATILSRRIAVEYSDSDDDSEFDENDWSD",
# DYN3_HUMAN
"MGNREMEELIPLVNRLQDAFSALGQSCLLELPQIAVVGGQSAGKSSVLENFVGRDFLPRGSGIVTRRPLVLQLVTSKAEYAEFLHCKGKKFTDFDEVRLEIEAETDRVTGMNKGISSIPINLRVYSPHVLNLTLIDLPGITKVPVGDQPPDIEYQIREMIMQFITRENCLILAVTPANTDLANSDALKLAKEVDPQGLRTIGVITKLDLMDEGTDARDVLENKLLPLRRGYVGVVNRSQKDIDGKKDIKAAMLAERKFFLSHPAYRHIADRMGTPHLQKVLNQQLTNHIRDTLPNFRNKLQGQLLSIEHEVEAYKNFKPEDPTRKTKALLQMVQQFAVDFEKRIEGSGDQVDTLELSGGAKINRIFHERFPFEIVKMEFNEKELRREISYAIKNIHGIRTGLFTPDMAFEAIVKKQIVKLKGPSLKSVDLVIQELINTVKKCTKKLANFPRLCEETERIVANHIREREGKTKDQVLLLIDIQVSYINTNHEDFIGFANAQQRSSQVHKKTTVGNQGTNLPPSRQIVIRKGWLTISNIGIMKGGSKGYWFVLTAESLSWYKDDEEKEKKYMLPLDNLKVRDVEKSFMSSKHIFALFNTEQRNVYKDYRFLELACDSQEDVDSWKASLLRAGVYPDKSVAENDENGQAENFSMDPQLERQVETIRNLVDSYMSIINKCIRDLIPKTIMHLMINNVKDFINSELLAQLYSSEDQNTLMEESAEQAQRRDEMLRMYQALKEALGIIGDISTATVSTPAPPPVDDSWIQHSRRSPPPSPTTQRRPTLSAPLARPTSGRGPAPAIPSPGPHSGAPPVPFRPGPLPPFPSSSDSFGAPPQVPSRPTRAPPSVPSRRPPPSPTRPTIIRPLESSLLD",
# RIMS2_HUMAN
"MSAPVGPRGRLAPIPAASQPPLQPEMPDLSHLTEEERKIILAVMDRQKKKVKEEHKPQLTQWFPFSGITELVNNVLQPQQKQQNEKEPQTKLHQQFEMYKEQVKKMGEESQQQQEQKGDAPTCGICHKTKFADGCGHNCSYCQTKFCARCGGRVSLRSNKVMWVCNLCRKQQEILTKSGAWFYNSGSNTPQQPDQKVLRGLRNEEAPQEKKPKLHEQTQFQGPSGDLSVPAVEKSRSHGLTRQHSIKNGSGVKHHIASDIASDRKRSPSVSRDQNRRYDQREEREEYSQYATSDTAMPRSPSDYADRRSQHEPQFYEDSDHLSYRDSNRRSHRHSKEYIVDDEDVESRDEYERQRREEEYQSRYRSDPNLARYPVKPQPYEEQMRIHAEVSRARHERRHSDVSLANADLEDSRISMLRMDRPSRQRSISERRAAMENQRSYSMERTREAQGPSSYAQRTTNHSPPTPRRSPLPIDRPDLRRTDSLRKQHHLDPSSAVRKTKREKMETMLRNDSLSSDQSESVRPPPPKPHKSKKGGKMRQISLSSSEEELASTPEYTSCDDVEIESESVSEKGDSQKGKRKTSEQAVLSDSNTRSERQKEMMYFGGHSLEEDLEWSEPQIKDSGVDTCSSTTLNEEHSHSDKHPVTWQPSKDGDRLIGRILLNKRLKDGSVPRDSGAMLGLKVVGGKMTESGRLCAFITKVKKGSLADTVGHLRPGDEVLEWNGRLLQGATFEEVYNIILESKPEPQVELVVSRPIGDIPRIPDSTHAQLESSSSSFESQKMDRPSISVTSPMSPGMLRDVPQFLSGQLSIKLWFDKVGHQLIVTILGAKDLPSREDGRPRNPYVKIYFLPDRSDKNKRRTKTVKKTLEPKWNQTFIYSPVHRREFRERMLEITLWDQARVREEESEFLGEILIELETALLDDEPHWYKLQTHDVSSLPLPHPSPYMPRRQLHGESPTRRLQRSKRISDSEVSDYDCDDGIGVVSDYRHDGRDLQSSTLSVPEQVMSSNHCSPSGSPHRVDVIGRTRSWSPSVPPPQSRNVEQGLRGTRTMTGHYNTISRMDRHRVMDDHYSPDRDRDCEAADRQPYHRSRSTEQRPLLERTTTRSRSTERPDTNLMRSMPSLMTGRSAPPSPALSRSHPRTGSVQTSPSSTPVAGRRGRQLPQLPPKGTLDRKAGGKKLRSTVQRSTETGLAVEMRNWMTRQASRESTDGSMNSYSSEGNLIFPGVRLASDSQFSDFLDGLGPAQLVGRQTLATPAMGDIQVGMMDKKGQLEVEIIRARGLVVKPGSKTLPAPYVKVYLLDNGVCIAKKKTKVARKTLEPLYQQLLSFEESPQGKVLQIIVWGDYGRMDHKSFMGVAQILLDELELSNMVIGWFKLFPPSSLVDPTLAPLTRRASQSSLESSTGPSYSRS",
# CNTN6_HUMAN
"MRLLWKLVILLPLINSSAGDGLLSRPIFTQEPHDVIFPLDLSKSEVILNCAANGYPSPHYRWKQNGTDIDFTMSYHYRLDGGSLAINSPHTDQDIGMYQCLATNLLGTILSRKAKLQFAYIEDFETKTRSTVSVREGQGVVLLCGPPPHFGDLSYAWTFNDNPLYVQEDNRRFVSQETGNLYIAKVEPSDVGNYTCFITNKEAQRSVQGPPTPLVQRTDGVMGEYEPKIEVRFPETIQAAKDSSVKLECFALGNPVPDISWRRLDGSPLPGKVKYSKSQAILEIPNFQQEDEGFYECIASNLRGRNLAKGQLIFYAPPEWEQKIQNTHLSIYDNLLWECKASGKPNPWYTWLKNGERLNPEERIQIENGTLIITMLNVSDSGVYQCAAENKYQIIYANAELRVLASAPDFSKSPVKKKSFVQVGGDIVIGCKPNAFPRAAISWKRGTETLRQSKRIFLLEDGSLKIYNITRSDAGSYTCIATNQFGTAKNTGSLIVKERTVITVPPSKMDVTVGESIVLPCQVSHDPSIEVVFVWFFNGDVIDLKKGVAHFERIGGESVGDLMIRNIQLHHSGKYLCTVQTTLESLSAVADIIVRGPPGPPEDVQVEDISSTTSQLSWRAGPDNNSPIQIFTIQTRTPFSVGWQAVATVPEILNGKTYNATVVGLSPWVEYEFRVVAGNSIGIGEPSEPSELLRTKASVPVVAPVNIHGGGGSRSELVITWESIPEELQNGEGFGYIIMFRPVGSTTWSKEKVSSVESSRFVYRNESIIPLSPFEVKVGVYNNEGEGSLSTVTIVYSGEDEPQLAPRGTSLQSFSASEMEVSWNAIAWNRNTGRVLGYEVLYWTDDSKESMIGKIRVSGNVTTKNITGLKANTIYFASVRAYNTAGTGPSSPPVNVTTKKSPPSQPPANIAWKLTNSKLCLNWEHVKTMENESEVLGYKILYRQNRQSKTHILETNNTSAELLVPFEEDYLIEIRTVSDGGDGSSSEEIRIPKMSSLSSRGIQFLEPSTHFLSIVIVIFHCFAIQPLI",
# CTND2_HUMAN
"MFARKPPGAAPLGAMPVPDQPSSASEKTSSLSPGLNTSNGDGSETETTSAILASVKEQELQFERLTRELEAERQIVASQLERCKLGSETGSMSSMSSAEEQFQWQSQDGQKDIEDELTTGLELVDSCIRSLQESGILDPQDYSTGERPSLLSQSALQLNSKPEGSFQYPASYHSNQTLALGETTPSQLPARGTQARATGQSFSQGTTSRAGHLAGPEPAPPPPPPPREPFAPSLGSAFHLPDAPPAAAAAALYYSSSTLPAPPRGGSPLAAPQGGSPTKLQRGGSAPEGATYAAPRGSSPKQSPSRLAKSYSTSSPINIVVSSAGLSPIRVTSPPTVQSTISSSPIHQLSSTIGTYATLSPTKRLVHASEQYSKHSQELYATATLQRPGSLAAGSRASYSSQHGHLGPELRALQSPEHHIDPIYEDRVYQKPPMRSLSQSQGDPLPPAHTGTYRTSTAPSSPGVDSVPLQRTGSQHGPQNAAAATFQRASYAAGPASNYADPYRQLQYCPSVESPYSKSGPALPPEGTLARSPSIDSIQKDPREFGWRDPELPEVIQMLQHQFPSVQSNAAAYLQHLCFGDNKIKAEIRRQGGIQLLVDLLDHRMTEVHRSACGALRNLVYGKANDDNKIALKNCGGIPALVRLLRKTTDLEIRELVTGVLWNLSSCDALKMPIIQDALAVLTNAVIIPHSGWENSPLQDDRKIQLHSSQVLRNATGCLRNVSSAGEEARRRMRECDGLTDALLYVIQSALGSSEIDSKTVENCVCILRNLSYRLAAETSQGQHMGTDELDGLLCGEANGKDAESSGCWGKKKKKKKSQDQWDGVGPLPDCAEPPKGIQMLWHPSIVKPYLTLLSECSNPDTLEGAAGALQNLAAGSWKWSVYIRAAVRKEKGLPILVELLRIDNDRVVCAVATALRNMALDVRNKELIGKYAMRDLVHRLPGGNNSNNTASKAMSDDTVTAVCCTLHEVITKNMENAKALRDAGGIEKLVGISKSKGDKHSPKVVKAASQVLNSMWQYRDLRSLYKKDGWSQYHFVASSSTIERDRQRPYSSSRTPSISPVRVSPNNRSASAPASPREMISLKERKTDYECTGSNATYHGAKGEHTSRKDAMTAQNTGISTLYRNSYGAPAEDIKHNQVSAQPVPQEPSRKDYETYQPFQNSTRNYDESFFEDQVHHRPPASEYTMHLGLKSTGNYVDFYSAARPYSELNYETSHYPASPDSWV",
# BAIP2_HUMAN
"MSLSRSEEMHRLTENVYKTIMEQFNPSLRNFIAMGKNYEKALAGVTYAAKGYFDALVKMGELASESQGSKELGDVLFQMAEVHRQIQNQLEEMLKSFHNELLTQLEQKVELDSRYLSAALKKYQTEQRSKGDALDKCQAELKKLRKKSQGSKNPQKYSDKELQYIDAISNKQGELENYVSDGYKTALTEERRRFCFLVEKQCAVAKNSAAYHSKGKELLAQKLPLWQQACADPSKIPERAVQLMQQVASNGATLPSALSASKSNLVISDPIPGAKPLPVPPELAPFVGRMSAQESTPIMNGVTGPDGEDYSPWADRKAAQPKSLSPPQSQSKLSDSYSNTLPVRKSVTPKNSYATTENKTLPRSSSMAAGLERNGRMRVKAIFSHAAGDNSTLLSFKEGDLITLLVPEARDGWHYGESEKTKMRGWFPFSYTRVLDSDGSDRLHMSLQQGKSSSTGNLLDKDDLAIPPPDYGAASRAFPAQTASGFKQRPYSVAVPAFSQGLDDYGARSMSRNPFAHVQLKPTVTNDRCDLSAQGPEGREHGDGSARTLAGR",
# SCN8A_HUMAN
"MAARLLAPPGPDSFKPFTPESLANIERRIAESKLKKPPKADGSHREDDEDSKPKPNSDLEAGKSLPFIYGDIPQGLVAVPLEDFDPYYLTQKTFVVLNRGKTLFRFSATPALYILSPFNLIRRIAIKILIHSVFSMIIMCTILTNCVFMTFSNPPDWSKNVEYTFTGIYTFESLVKIIARGFCIDGFTFLRDPWNWLDFSVIMMAYITEFVNLGNVSALRTFRVLRALKTISVIPGLKTIVGALIQSVKKLSDVMILTVFCLSVFALIGLQLFMGNLRNKCVVWPINFNESYLENGTKGFDWEEYINNKTNFYTVPGMLEPLLCGNSSDAGQCPEGYQCMKAGRNPNYGYTSFDTFSWAFLALFRLMTQDYWENLYQLTLRAAGKTYMIFFVLVIFVGSFYLVNLILAVVAMAYEEQNQATLEEAEQKEAEFKAMLEQLKKQQEEAQAAAMATSAGTVSEDAIEEEGEEGGGSPRSSSEISKLSSKSAKERRNRRKKRKQKELSEGEEKGDPEKVFKSESEDGMRRKAFRLPDNRIGRKFSIMNQSLLSIPGSPFLSRHNSKSSIFSFRGPGRFRDPGSENEFADDEHSTVEESEGRRDSLFIPIRARERRSSYSGYSGYSQGSRSSRIFPSLRRSVKRNSTVDCNGVVSLIGGPGSHIGGRLLPEATTEVEIKKKGPGSLLVSMDQLASYGRKDRINSIMSVVTNTLVEELEESQRKCPPCWYKFANTFLIWECHPYWIKLKEIVNLIVMDPFVDLAITICIVLNTLFMAMEHHPMTPQFEHVLAVGNLVFTGIFTAEMFLKLIAMDPYYYFQEGWNIFDGFIVSLSLMELSLADVEGLSVLRSFRLLRVFKLAKSWPTLNMLIKIIGNSVGALGNLTLVLAIIVFIFAVVGMQLFGKSYKECVCKINQDCELPRWHMHDFFHSFLIVFRVLCGEWIETMWDCMEVAGQAMCLIVFMMVMVIGNLVVLNLFLALLLSSFSADNLAATDDDGEMNNLQISVIRIKKGVAWTKLKVHAFMQAHFKQREADEVKPLDELYEKKANCIANHTGADIHRNGDFQKNGNGTTSGIGSSVEKYIIDEDHMSFINNPNLTVRVPIAVGESDFENLNTEDVSSESDPEGSKDKLDDTSSSEGSTIDIKPEVEEVPVEQPEEYLDPDACFTEGCVQRFKCCQVNIEEGLGKSWWILRKTCFLIVEHNWFETFIIFMILLSSGALAFEDIYIEQRKTIRTILEYADKVFTYIFILEMLLKWTAYGFVKFFTNAWCWLDFLIVAVSLVSLIANALGYSELGAIKSLRTLRALRPLRALSRFEGMRVVVNALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKYHYCFNETSEIRFEIEDVNNKTECEKLMEGNNTEIRWKNVKINFDNVGAGYLALLQVATFKGWMDIMYAAVDSRKPDEQPKYEDNIYMYIYFVIFIIFGSFFTLNLFIGVIIDNFNQQKKKFGGQDIFMTEEQKKYYNAMKKLGSKKPQKPIPRPLNKIQGIVFDFVTQQAFDIVIMMLICLNMVTMMVETDTQSKQMENILYWINLVFVIFFTCECVLKMFALRHYYFTIGWNIFDFVVVILSIVGMFLADIIEKYFVSPTLFRVIRLARIGRILRLIKGAKGIRTLLFALMMSLPALFNIGLLLFLVMFIFSIFGMSNFAYVKHEAGIDDMFNFETFGNSMICLFQITTSAGWDGLLLPILNRPPDCSLDKEHPGSGFKGDCGNPSVGIFFFVSYIIISFLIVVNMYIAIILENFSVATEESADPLSEDDFETFYEIWEKFDPDATQFIEYCKLADFADALEHPLRVPKPNTIELIAMDLPMVSGDRIHCLDILFAFTKRVLGDSGELDILRQQMEERFVASNPSKVSYEPITTTLRRKQEEVSAVVLQRAYRGHLARRGFICKKTTSNKLENGGTHREKKESTPSTASLPSYDSVTKPEKEKQQRAEEGRRERAKRQKEVRESKC",
# KCC2A_HUMAN
"MATITCTRFTEEYQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGHHYLIFDLVTGGELFEDIVAREYYSEADASHCIQQILEAVLHCHQMGVVHRDLKPENLLLASKLKGAAVKLADFGLAIEVEGEQQAWFGFAGTPGYLSPEVLRKDPYGKPVDLWACGVILYILLVGYPPFWDEDQHRLYQQIKAGAYDFPSPEWDTVTPEAKDLINKMLTINPSKRITAAEALKHPWISHRSTVASCMHRQETVDCLKKFNARRKLKGAILTTMLATRNFSGGKSGGNKKSDGVKESSESTNTTIEDEDTKVRKQEIIKVTEQLIEAISNGDFESYTKMCDPGMTAFEPEALGNLVEGLDFHRFYFENLWSRNSKPVHTTILNPHIHLMGDESACIAYIRITQYLDAGGIPRTAQSEETRVWHRRDGKWQIVHFHRSGAPSVLPH",
# CHM2B_HUMAN
"MASLFKKKTVDDVIKEQNRELRGTQRAIIRDRAALEKQEKQLELEIKKMAKIGNKEACKVLAKQLVHLRKQKTRTFAVSSKVTSMSTQTKVMNSQMKMAGAMSTTAKTMQAVNKKMDPQKTLQTMQNFQKENMKMEMTEEMINDTLDDIFDGSDDEEESQDIVNQVLDEIGIEISGKMAKAPSAARSLPSASTSKATISDEEIERQLKALGVD",
# PDE10_HUMAN
"MASLEEPLAPRPQGPLPAAGDEPGCGPGKLRPEPRLSAAGGGSAAGPGPAPEWPGRGRAERAAPPRPPLSSAGRPSPAGGPGALSARGGGCGWVAARAPLALAFSSRVPSSSPSFFYFWPPPPPPPPSFLPSSSAFHLPVRLPGREGAAAAAAAGGGGDAGGGGGGGQEAAPLSVPTSSSHRGGGGSGGGRRRLFLSPALQGLLLPARAGPRPPPPPRLPLGQAARRAGSPGFPGAGPGGGGQTPRRPQGASFALAAAAALLFGSDMEDGPSNNASCFRRLTECFLSPSLTDEKVKAYLSLHPQVLDEFVSESVSAETVEKWLKRKNNKSEDESAPKEVSRYQDTNMQGVVYELNSYIEQRLDTGGDNQLLLYELSSIIKIATKADGFALYFLGECNNSLCIFTPPGIKEGKPRLIPAGPITQGTTVSAYVAKSRKTLLVEDILGDERFPRGTGLESGTRIQSVLCLPIVTAIGDLIGILELYRHWGKEAFCLSHQEVATANLAWASVAIHQVQVCRGLAKQTELNDFLLDVSKTYFDNIVAIDSLLEHIMIYAKNLVNADRCALFQVDHKNKELYSDLFDIGEEKEGKPVFKKTKEIRFSIEKGIAGQVARTGEVLNIPDAYADPRFNREVDLYTGYTTRNILCMPIVSRGSVIGVVQMVNKISGSAFSKTDENNFKMFAVFCALALHCANMYHRIRHSECIYRVTMEKLSYHSICTSEEWQGLMQFTLPVRLCKEIELFHFDIGPFENMWPGIFVYMVHRSCGTSCFELEKLCRFIMSVKKNYRRVPYHNWKHAVTVAHCMYAILQNNHTLFTDLERKGLLIACLCHDLDHRGFSNSYLQKFDHPLAALYSTSTMEQHHFSQTVSILQLEGHNIFSTLSSSEYEQVLEIIRKAIIATDLALYFGNRKQLEEMYQTGSLNLNNQSHRDRVIGLMMTACDLCSVTKLWPVTKLTANDIYAEFWAEGDEMKKLGIQPIPMMDRDKKDEVPQGQLGFYNAVAIPCYTTLTQILPPTEPLLKACRDNLSQWEKVIRGEETATWISSPSVAQKAAASED",
# LZTS1_HUMAN
"MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGFSQDSGHGKSSSKMGKSEDFFYIKVSQKARGSHHPDYTALSSGDLGGQAGVDFDPSTPPKLMPFSNQLEMGSEKGAVRPTAFKPVLPRSGAILHSSPESASHQLHPAPPDKPKEQELKPGLCSGALSDSGRNSMSSLPTHSTSSSYQLDPLVTPVGPTSRFGGSAHNITQGIVLQDSNMMSLKALSFSDGGSKLGHSNKADKGPSCVRSPISTDECSIQELEQKLLEREGALQKLQRSFEEKELASSLAYEERPRRCRDELEGPEPKGGNKLKQASQKSQRAQQVLHLQVLQLQQEKRQLRQELESLMKEQDLLETKLRSYEREKTSFGPALEETQWEVCQKSGEISLLKQQLKESQTEVNAKASEILGLKAQLKDTRGKLEGLELRTQDLEGALRTKGLELEVCENELQRKKNEAELLREKVNLLEQELQELRAQAALARDMGPPTFPEDVPALQRELERLRAELREERQGHDQMSSGFQHERLVWKEEKEKVIQYQKQLQQSYVAMYQRNQRLEKALQQLARGDSAGEPLEVDLEGADIPYEDIIATEI",
# EIF3L_HUMAN
"MSYPADDYESEAAYDPYAYPSDYDMHTGDPKQDLAYERQYEQQTYQVIPEVIKNFIQYFHKTVSDLIDQKVYELQASRVSSDVIDQKVYEIQDIYENSWTKLTERFFKNTPWPEAEAIAPQVGNDAVFLILYKELYYRHIYAKVSGGPSLEQRFESYYNYCNLFNYILNADGPAPLELPNQWLWDIIDEFIYQFQSFSQYRCKTAKKSEEEIDFLRSNPKIWNVHSVLNVLHSLVDKSNINRQLEVYTSGGDPESVAGEYGRHSLYKMLGYFSLVGLLRLHSLLGDYYQAIKVLENIELNKKSMYSRVPECQVTTYYYVGFAYLMMRRYQDAIRVFANILLYIQRTKSMFQRTTYKYEMINKQNEQMHALLAIALTMYPMRIDESIHLQLREKYGDKMLRMQKGDPQVYEELFSYSCPKFLSPVVPNYDNVHPNYHKEPFLQQLKVFSDEVQQQAQLSTIRSFLKLYTTMPVAKLAGFLDLTEQEFRIQLLVFKHKMKNLVWTSGISALDGEFQSASEVDFYIDKDMIHIADTKVARRYGDFFIRQIHKFEELNRTLKKMGQRP",
# TPPC4_HUMAN
"MAIFSVYVVNKAGGLIYQLDSYAPRAEAEKTFSYPLDLLLKLHDERVLVAFGQRDGIRVGHAVLAINGMDVNGRYTADGKEVLEYLGNPANYPVSIRFGRPRLTSNEKLMLASMFHSLFAIGSQLSPEQGSSGIEMLETDTFKLHCYQTLTGIKFVVLADPRQAGIDSLLRKIYEIYSDFALKNPFYSLEMPIRCELFDQNLKLALEVAEKAGTFGPGS",
# NCKP1_HUMAN
"MSRSVLQPSQQKLAEKLTILNDRGVGMLTRLYNIKKACGDPKAKPSYLIDKNLESAVKFIVRKFPAVETRNNNQQLAQLQKEKSEILKNLALYYFTFVDVMEFKDHVCELLNTIDVCQVFFDITVNFDLTKNYLDLIITYTTLMILLSRIEERKAIIGLYNYAHEMTHGASDREYPRLGQMIVDYENPLKKMMEEFVPHSKSLSDALISLQMVYPRRNLSADQWRNAQLLSLISAPSTMLNPAQSDTMPCEYLSLDAMEKWIIFGFILCHGILNTDATALNLWKLALQSSSCLSLFRDEVFHIHKAAEDLFVNIRGYNKRINDIRECKEAAVSHAGSMHRERRKFLRSALKELATVLSDQPGLLGPKALFVFMALSFARDEIIWLLRHADNMPKKSADDFIDKHIAELIFYMEELRAHVRKYGPVMQRYYVQYLSGFDAVVLNELVQNLSVCPEDESIIMSSFVNTMTSLSVKQVEDGEVFDFRGMRLDWFRLQAYTSVSKASLGLADHRELGKMMNTIIFHTKMVDSLVEMLVETSDLSIFCFYSRAFEKMFQQCLELPSQSRYSIAFPLLCTHFMSCTHELCPEERHHIGDRSLSLCNMFLDEMAKQARNLITDICTEQCTLSDQLLPKHCAKTISQAVNKKSKKQTGKKGEPEREKPGVESMRKNRLVVTNLDKLHTALSELCFSINYVPNMVVWEHTFTPREYLTSHLEIRFTKSIVGMTMYNQATQEIAKPSELLTSVRAYMTVLQSIENYVQIDITRVFNNVLLQQTQHLDSHGEPTITSLYTNWYLETLLRQVSNGHIAYFPAMKAFVNLPTENELTFNAEEYSDISEMRSLSELLGPYGMKFLSESLMWHISSQVAELKKLVVENVDVLTQMRTSFDKPDQMAALFKRLSSVDSVLKRMTIIGVILSFRSLAQEALRDVLSYHIPFLVSSIEDFKDHIPRETDMKVAMNVYELSSAAGLPCEIDPALVVALSSQKSENISPEEEYKIACLLMVFVAVSLPTLASNVMSQYSPAIEGHCNNIHCLAKAINQIAAALFTIHKGSIEDRLKEFLALASSSLLKIGQETDKTTTRNRESVYLLLDMIVQESPFLTMDLLESCFPYVLLRNAYHAVYKQSVTSSA",
# DLGP4_HUMAN
"MKGLGDSRPRHLSDSLDPPHEPLFAGTDRNPYLLSPTEAFAREARFPGQNTLPGDGLFPLNNQLPPPSSTFPRIHYNSHFEVPEESPFPSHAQATKINRLPANLLDQFEKQLPIHRDGFSTLQFPRGEAKARGESPGRIRHLVHSVQRLFFTKAPSLEGTAGKVGGNGSKKGGMEDGKGRRAKSKERAKAGEPKRRSRSNISGWWSSDDNLDGEAGAFRSSGPASGLMTLGRQAERSQPRYFMHAYNTISGHMLKTTKNNTTELTAPPPPPAPPATCPSLGVGTDTNYVKRGSWSTLTLSHAHEVCQKTSATLDKSLLKSKSCHQGLAYHYLQVPGGGGEWSTTLLSPRETDAAAEGPIPCRRMRSGSYIKAMGDEDSDESGGSPKPSPKTAARRQSYLRATQQSLGEQSNPRRSLDRLDSVDMLLPSKCPSWEEDYTPVSDSLNDSSCISQIFGQASLIPQLFGHEQQVREAELSDQYEAACESACSEAESTAAETLDLPLPSYFRSRSHSYLRAIQAGCSQEEDSVSLQSLSPPPSTGSLSNSRTLPSSSCLVAYKKTPPPVPPRTTSKPFISVTVQSSTESAQDTYLDSQDHKSEVTSQSGLSNSSDSLDSSTRPPSVTRGGVAPAPEAPEPPPKHAALKSEQGTLTSSESHPEAAPKRKLSSIGIQVDCIQPVPKEEPSPATKFQSIGVQVEDDWRSSVPSHSMSSRRDTDSDTQDANDSSCKSSERSLPDCTPHPNSISIDAGPRQAPKIAQIKRNLSYGDNSDPALEASSLPPPDPWLETSSSSPAEPAQPGACRRDGYWFLKLLQAETERLEGWCCQMDKETKENNLSEEVLGKVLSAVGSAQLLMSQKFQQFRGLCEQNLNPDANPRPTAQDLAGFWDLLQLSIEDISMKFDELYHLKANSWQLVETPEKRKEEKKPPPPVPKKPAKSKPAVSRDKASDASDKQRQEARKRLLAAKRAASVRQNSATESADSIEIYVPEAQTRL",
# ST38L_HUMAN
"MAMTAGTTTTFPMSNHTRERVTVAKLTLENFYSNLILQHEERETRQKKLEVAMEEEGLADEEKKLRRSQHARKETEFLRLKRTRLGLDDFESLKVIGRGAFGEVRLVQKKDTGHIYAMKILRKSDMLEKEQVAHIRAERDILVEADGAWVVKMFYSFQDKRNLYLIMEFLPGGDMMTLLMKKDTLTEEETQFYISETVLAIDAIHQLGFIHRDIKPDNLLLDAKGHVKLSDFGLCTGLKKAHRTEFYRNLTHNPPSDFSFQNMNSKRKAETWKKNRRQLAYSTVGTPDYIAPEVFMQTGYNKLCDWWSLGVIMYEMLIGYPPFCSETPQETYRKVMNWKETLVFPPEVPISEKAKDLILRFCIDSENRIGNSGVEEIKGHPFFEGVDWEHIRERPAAIPIEIKSIDDTSNFDDFPESDILQPVPNTTEPDYKSKDWVFLNYTYKRFEGLTQRGSIPTYMKAGKL",
# NTNG1_HUMAN
"MYLSRFLSIHALWVTVSSVMQPYPLVWGHYDLCKTQIYTEEGKVWDYMACQPESTDMTKYLKVKLDPPDITCGDPPETFCAMGNPYMCNNECDASTPELAHPPELMFDFEGRHPSTFWQSATWKEYPKPLQVNITLSWSKTIELTDNIVITFESGRPDQMILEKSLDYGRTWQPYQYYATDCLDAFHMDPKSVKDLSQHTVLEIICTEEYSTGYTTNSKIIHFEIKDRFAFFAGPRLRNMASLYGQLDTTKKLRDFFTVTDLRIRLLRPAVGEIFVDELHLARYFYAISDIKVRGRCKCNLHATVCVYDNSKLTCECEHNTTGPDCGKCKKNYQGRPWSPGSYLPIPKGTANTCIPSISSIGNCECFGHSNRCSYIDLLNTVICVSCKHNTRGQHCELCRLGYFRNASAQLDDENVCIECYCNPLGSIHDRCNGSGFCECKTGTTGPKCDECLPGNSWHYGCQPNVCDNELLHCQNGGTCHNNVRCLCPAAYTGILCEKLRCEEAGSCGSDSGQGAPPHGSPALLLLTTLLGTASPLVF",
# RP3A_HUMAN
"MTDTVFSNSSNRWMYPSDRPLQSNDKEQLQAGWSVHPGGQPDRQRKQEELTDEEKEIINRVIARAEKMEEMEQERIGRLVDRLENMRKNVAGDGVNRCILCGEQLGMLGSACVVCEDCKKNVCTKCGVETNNRLHSVWLCKICIEQREVWKRSGAWFFKGFPKQVLPQPMPIKKTKPQQPVSEPAAPEQPAPEPKHPARAPARGDSEDRRGPGQKTGPDPASAPGRGNYGPPVRRASEARMSSSSRDSESWDHSGGAGDSSRSPAGLRRANSVQASRPAPGSVQSPAPPQPGQPGTPGGSRPGPGPAGRFPDQKPEVAPSDPGTTAPPREERTGGVGGYPAVGAREDRMSHPSGPYSQASAAAPQPAAARQPPPPEEEEEEANSYDSDEATTLGALEFSLLYDQDNSSLQCTIIKAKGLKPMDSNGLADPYVKLHLLPGASKSNKLRTKTLRNTRNPIWNETLVYHGITDEDMQRKTLRISVCDEDKFGHNEFIGETRFSLKKLKPNQRKNFNICLERVIPMKRAGTTGSARGMALYEEEQVERVGDIEERGKILVSLMYSTQQGGLIVGIIRCVHLAAMDANGYSDPFVKLWLKPDMGKKAKHKTQIKKKTLNPEFNEEFFYDIKHSDLAKKSLDISVWDYDIGKSNDYIGGCQLGISAKGERLKHWYECLKNKDKKIERWHQLQNENHVSSD",
# E41L3_HUMAN
"MTTESGSDSESKPDQEAEPQEAAGAQGRAGAPVPEPPKEEQQQALEQFAAAAAHSTPVRREVTDKEQEFAARAAKQLEYQQLEDDKLSQKSSSSKLSRSPLKIVKKPKSMQCKVILLDGSEYTCDVEKRSRGQVLFDKVCEHLNLLEKDYFGLTYRDAENQKNWLDPAKEIKKQVRSGAWHFSFNVKFYPPDPAQLSEDITRYYLCLQLRDDIVSGRLPCSFVTLALLGSYTVQSELGDYDPDECGSDYISEFRFAPNHTKELEDKVIELHKSHRGMTPAEAEMHFLENAKKLSMYGVDLHHAKDSEGVEIMLGVCASGLLIYRDRLRINRFAWPKVLKISYKRNNFYIKIRPGEFEQFESTIGFKLPNHRAAKRLWKVCVEHHTFFRLLLPEAPPKKFLTLGSKFRYSGRTQAQTRRASALIDRPAPYFERSSSKRYTMSRSLDGEVGTGQYATTKGISQTNLITTVTPEKKAEEERDEEEDKRRKGEEVTPISAIRHEGKSPGLGTDSCPLSPPSTHCAPTSPTELRRRCKENDCKLPGYEPSRAEHLPGEPALDSDGPGRPYLGDQDVAFSYRQQTGKGTTLFSFSLQLPESFPSLLDDDGYLSFPNLSETNLLPQSLQHYLPIRSPSLVPCFLFIFFFLLSASFSVPYALTLSFPLALCLCYLEPKAASLSASLDNDPSDSSEEETDSERTDTAADGETTATESDQEEDAELKAQELEKTQDDLMKHQTNISELKRTFLETSTDTAVTNEWEKRLSTSPVRLAARQEDAPMIEPLVPEETKQSSGEKLMDGSEIFSLLESARKPTEFIGGVTSTSQSWVQKMETKTESSGIETEPTVHHLPLSTEKVVQETVLVEERRVVHASGDASYSAGDSGDAAAQPAFTGIKGKEGSALTEGAKEEGGEEVAKAVLEQEETAAASRERQEEQSAAIHISETLEQKPHFESSTVKTETISFGSVSPGGVKLEISTKEVPVVHTETKTITYESSQVDPGTDLEPGVLMSAQTITSETTSTTTTTHITKTVKGGISETRIEKRIVITGDADIDHDQALAQAIKEAKEQHPDMSVTKVVVHKETEITPEDGED",
# STB5L_HUMAN
"MKKFNFRKVLDGLTASSPGSGSSSGSNSGGGAGSGSVHPAGTAGVLREEIQETLTSEYFQICKTVRHGFPHQPTALAFDPVQKILAIGTRTGAIRILGRPGVDCYCQHESGAAVLQLQFLINEGALVSASSDDTLHLWNLRQKRPAILHSLKFNRERITYCHLPFQSKWLYVGTERGNTHIVNIESFILSGYVIMWNKAIELSTKTHPGPVVHLSDSPRDEGKLLIGYENGTVVFWDLKSKRAELRVYYDEAIHSIDWHHEGKQFMCSHSDGSLTLWNLKSPSRPFQTTIPHGKSQREGRKSESCKPILKVEYKTCKNSEPFIIFSGGLSYDKACRRPSLTIMHGKAITVLEMDHPIVEFLTLCETPYPNEFQEPYAVVVLLEKDLIVVDLTQSNFPIFENPYPMDIHESPVTCTAYFADCPPDLILVLYSIGVKHKKQGYSNKEWPISGGAWNLGAQTYPEIIITGHADGSIKFWDASAITLQMLYKLKTSKVFEKQKVGEGKQTCEIVEEDPFAIQMIYWCPESRIFCVSGVSAYVIIYKFSRHEITTEIVSLEVRLQYDVEDIITPEPETSPPFPDLSAQLPSSRSLSGSTNTVASEGVTKDSIPCLNVKTRPVRMPPGYQAELVIQLVWVDGEPPQQITSLAVSSAYGIVAFGNCNGLAVVDFIQKTVLLSMGTIDLYRSSDLYQRQPRSPRKNKQFIADNFCMRGLSNFYPDLTKRIRTSYQSLTELNDSPVPLELERCKSPTSDHVNGHCTSPTSQSCSSGKRLSSADVSKVNRWGPGRPPFRKAQSAACMEISLPVTTEENRENSYNRSRSSSISSIDKDSKEAITALYFMDSFARKNDSTISPCLFVGTSLGMVLIISLNLPLADEQRFTEPVMVLPSGTFLSLKGAVLTFSCMDRMGGLMQPPYEVWRDPNNIDENEKSWRRKVVMNSSSASQEIGDHQYTIICSEKQAKVFSLPSQTCLYVHNITETSFILQANVVVMCSSACLACFCANGHIMIMSLPSLRPMLDVNYLPLTDMRIARTFCFTNEGQALYLVSPTEIQRLTYSQEMCDNLQDMLGDLFTPIETPEAQNRGFLKGLFGGSGQTFDREELFGEASAGKASRSLAQHIPGPGSIEGMKGAAGGVMGELTRARIALDERGQRLGELEEKTAGMMTSAEAFSKHAHELMLKYKDKKWYQF",
# AT8A1_HUMAN
"MPTMRRTVSEIRSRAEGYEKTDDVSEKTSLADQEEVRTIFINQPQLTKFCNNHVSTAKYNIITFLPRFLYSQFRRAANSFFLFIALLQQIPDVSPTGRYTTLVPLLFILAVAAIKEIIEDIKRHKADNAVNKKQTQVLRNGAWEIVHWEKVAVGEIVKVTNGEHLPADLISLSSSEPQAMCYIETSNLDGETNLKIRQGLPATSDIKDVDSLMRISGRIECESPNRHLYDFVGNIRLDGHGTVPLGADQILLRGAQLRNTQWVHGIVVYTGHDTKLMQNSTSPPLKLSNVERITNVQILILFCILIAMSLVCSVGSAIWNRRHSGKDWYLNLNYGGASNFGLNFLTFIILFNNLIPISLLVTLEVVKFTQAYFINWDLDMHYEPTDTAAMARTSNLNEELGQVKYIFSDKTGTLTCNVMQFKKCTIAGVAYGHVPEPEDYGCSPDEWQNSQFGDEKTFSDSSLLENLQNNHPTAPIICEFLTMMAVCHTAVPEREGDKIIYQAASPDEGALVRAAKQLNFVFTGRTPDSVIIDSLGQEERYELLNVLEFTSARKRMSVIVRTPSGKLRLYCKGADTVIYDRLAETSKYKEITLKHLEQFATEGLRTLCFAVAEISESDFQEWRAVYQRASTSVQNRLLKLEESYELIEKNLQLLGATAIEDKLQDQVPETIETLMKADIKIWILTGDKQETAINIGHSCKLLKKNMGMIVINEGSLDGTRETLSRHCTTLGDALRKENDFALIIDGKTLKYALTFGVRQYFLDLALSCKAVICCRVSPLQKSEVVEMVKKQVKVVTLAIGDGANDVSMIQTAHVGVGISGNEGLQAANSSDYSIAQFKYLKNLLMIHGAWNYNRVSKCILYCFYKNIVLYIIEIWFAFVNGFSGQILFERWCIGLYNVMFTAMPPLTLGIFERSCRKENMLKYPELYKTSQNALDFNTKVFWVHCLNGLFHSVILFWFPLKALQYGTAFGNGKTSDYLLLGNFVYTFVVITVCLKAGLETSYWTWFSHIAIWGSIALWVVFFGIYSSLWPAIPMAPDMSGEAAMLFSSGVFWMGLLFIPVASLLLDVVYKVIKRTAFKTLVDEVQELEAKSQDPGAVVLGKSLTERAQLLKNVFKKNHVNLYRSESLQQNLLHGYAFSQDENGIVSQSEVIRAYDTTKQRPDEW",
# AP3M1_HUMAN
"MIHSLFLINCSGDIFLEKHWKSVVSQSVCDYFFEAQEKAADVENVPPVISTPHHYLISIYRDKLFFVSVIQTEVPPLFVIEFLHRVADTFQDYFGECSEAAIKDNVVIVYELLEEMLDNGFPLATESNILKELIKPPTILRSVVNSITGSSNVGDTLPTGQLSNIPWRRAGVKYTNNEAYFDVVEEIDAIIDKSGSTVFAEIQGVIDACIKLSGMPDLSLSFMNPRLLDDVSFHPCIRFKRWESERVLSFIPPDGNFRLISYRVSSQNLVAIPVYVKHSISFKENSSCGRFDITIGPKQNMGKTIEGITVTVHMPKVVLNMNLTPTQGSYTFDPVTKVLTWDVGKITPQKLPSLKGLVNLQSGAPKPEENPSLNIQFKIQQLAISGLKVNRLDMYGEKYKPFKGVKYVTKAGKFQVRT",
# GUAD_HUMAN
"MCAAQMPPLAHIFRGTFVHSTWTCPMEVLRDHLLGVSDSGKIVFLEEASQQEKLAKEWCFKPCEIRELSHHEFFMPGLVDTHIHASQYSFAGSSIDLPLLEWLTKYTFPAEHRFQNIDFAEEVYTRVVRRTLKNGTTTACYFATIHTDSSLLLADITDKFGQRAFVGKVCMDLNDTFPEYKETTEESIKETERFVSEMLQKNYSRVKPIVTPRFSLSCSETLMGELGNIAKTRDLHIQSHISENRDEVEAVKNLYPSYKNYTSVYDKNNLLTNKTVMAHGCYLSAEELNVFHERGASIAHCPNSNLSLSSGFLNVLEVLKHEVKIGLGTDVAGGYSYSMLDAIRRAVMVSNILLINKVNEKSLTLKEVFRLATLGGSQALGLDGEIGNFEVGKEFDAILINPKASDSPIDLFYGDFFGDISEAVIQKFLYLGDDRNIEEVYVGGKQVVPFSSSV",
# GIT1_HUMAN
"MSRKGPRAEVCADCSAPDPGWASISRGVLVCDECCSVHRSLGRHISIVKHLRHSAWPPTLLQMVHTLASNGANSIWEHSLLDPAQVQSGRRKANPQDKVHPIKSEFIRAKYQMLAFVHKLPCRDDDGVTAKDLSKQLHSSVRTGNLETCLRLLSLGAQANFFHPEKGTTPLHVAAKAGQTLQAELLVVYGADPGSPDVNGRTPIDYARQAGHHELAERLVECQYELTDRLAFYLCGRKPDHKNGHYIIPQMADSLDLSELAKAAKKKLQALSNRLFEELAMDVYDEVDRRENDAVWLATQNHSTLVTERSAVPFLPVNPEYSATRNQGRQKLARFNAREFATLIIDILSEAKRRQQGKSLSSPTDNLELSLRSQSDLDDQHDYDSVASDEDTDQEPLRSTGATRSNRARSMDSSDLSDGAVTLQEYLELKKALATSEAKVQQLMKVNSSLSDELRRLQREIHKLQAENLQLRQPPGPVPTPPLPSERAEHTPMAPGGSTHRRDRQAFSMYEPGSALKPFGGPPGDELTTRLQPFHSTELEDDAIYSVHVPAGLYRIRKGVSASAVPFTPSSPLLSCSQEGSRHTSKLSRHGSGADSDYENTQSGDPLLGLEGKRFLELGKEEDFHPELESLDGDLDPGLPSTEDVILKTEQVTKNIQELLRAAQEFKHDSFVPCSEKIHLAVTEMASLFPKRPALEPVRSSLRLLNASAYRLQSECRKTVPPEPGAPVDFQLLTQQVIQCAYDIAKAAKQLVTITTREKKQ",
# NSG2_HUMAN
"MVKLNSNPSEKGTKPPSVEDGFQTVPLITPLEVNHLQLPAPEKVIVKTRTEYQPEQKNKGKFRVPKIAEFTVTILVSLALAFLACIVFLVVYKAFTYDHSCPEGFVYKHKRCIPASLDAYYSSQDPNSRSRFYTVISHYSVAKQSTARAIGPWLSAAAVIHEPKPPKTQGH",
# SC6A5_HUMAN
"MDCSAPKEMNKLPANSPEAAAAQGHPDGPCAPRTSPEQELPAAAAPPPPRVPRSASTGAQTFQSADARACEAERPGVGSCKLSSPRAQAASAALRDLREAQGAQASPPPGSSGPGNALHCKIPFLRGPEGDANVSVGKGTLERNNTPVVGWVNMSQSTVVLATDGITSVLPGSVATVATQEDEQGDENKARGNWSSKLDFILSMVGYAVGLGNVWRFPYLAFQNGGGAFLIPYLMMLALAGLPIFFLEVSLGQFASQGPVSVWKAIPALQGCGIAMLIISVLIAIYYNVIICYTLFYLFASFVSVLPWGSCNNPWNTPECKDKTKLLLDSCVISDHPKIQIKNSTFCMTAYPNVTMVNFTSQANKTFVSGSEEYFKYFVLKISAGIEYPGEIRWPLALCLFLAWVIVYASLAKGIKTSGKVVYFTATFPYVVLVILLIRGVTLPGAGAGIWYFITPKWEKLTDATVWKDAATQIFFSLSAAWGGLITLSSYNKFHNNCYRDTLIVTCTNSATSIFAGFVIFSVIGFMANERKVNIENVADQGPGIAFVVYPEALTRLPLSPFWAIIFFLMLLTLGLDTMFATIETIVTSISDEFPKYLRTHKPVFTLGCCICFFIMGFPMITQGGIYMFQLVDTYAASYALVIIAIFELVGISYVYGLQRFCEDIEMMIGFQPNIFWKVCWAFVTPTILTFILCFSFYQWEPMTYGSYRYPNWSMVLGWLMLACSVIWIPIMFVIKMHLAPGRFIERLKLVCSPQPDWGPFLAQHRGERYKNMIDPLGTSSLGLKLPVKDLELGTQC",
# PHOCN_HUMAN
"MVMAEGTAVLRRNRPGTKAQDFYNWPDESFDEMDSTLAVQQYIQQNIRADCSNIDKILEPPEGQDEGVWKYEHLRQFCLELNGLAVKLQSECHPDTCTQMTATEQWIFLCAAHKTPKECPAIDYTRHTLDGAACLLNSNKYFPSRVSIKESSVAKLGSVCRRIYRIFSHAYFHHRQIFDEYENETFLCHRFTKFVMKYNLMSKDNLIVPILEEEVQNSVSGESEA",
# GRIP1_HUMAN
"MIAVSFKCRCQILRRLTKDESPYTKSASQTKPPDGALAVRRQSIPEEFKGSTVVELMKKEGTTLGLTVSGGIDKDGKPRVSNLRQGGIAARSDQLDVGDYIKAVNGINLAKFRHDEIISLLKNVGERVVLEVEYELPPVSVQGSSVIFRTVEVTLHKEGNTFGFVIRGGAHDDRNKSRPVVITCVRPGGPADREGTIKPGDRLLSVDGIRLLGTTHAEAMSILKQCGQEAALLIEYDVSVMDSVATASGPLLVEVAKTPGASLGVALTTSMCCNKQVIVIDKIKSASIADRCGALHVGDHILSIDGTSMEYCTLAEATQFLANTTDQVKLEILPHHQTRLALKGPDHVKIQRSDRQLTWDSWASNHSSLHTNHHYNTYHPDHCRVPALTFPKAPPPNSPPALVSSSFSPTSMSAYSLSSLNMGTLPRSLYSTSPRGTMMRRRLKKKDFKSSLSLASSTVGLAGQVVHTETTEVVLTADPVTGFGIQLQGSVFATETLSSPPLISYIEADSPAERCGVLQIGDRVMAINGIPTEDSTFEEASQLLRDSSITSKVTLEIEFDVAESVIPSSGTFHVKLPKKHNVELGITISSPSSRKPGDPLVISDIKKGSVAHRTGTLELGDKLLAIDNIRLDNCSMEDAVQILQQCEDLVKLKIRKDEDNSDEQESSGAIIYTVELKRYGGPLGITISGTEEPFDPIIISSLTKGGLAERTGAIHIGDRILAINSSSLKGKPLSEAIHLLQMAGETVTLKIKKQTDAQSASSPKKFPISSHLSDLGDVEEDSSPAQKPGKLSDMYPSTVPSVDSAVDSWDGSAIDTSYGTQGTSFQASGYNFNTYDWRSPKQRGSLSPVTKPRSQTYPDVGLSYEDWDRSTASGFAGAADSAETEQEENFWSQALEDLETCGQSGILRELEEKADRRVSLRNMTLLATIMSGSTMSLNHEAPTPRSQLGRQASFQERSSSRPHYSQTTRSNTLPSDVGRKSVTLRKMKQEIKEIMSPTPVELHKVTLYKDSDMEDFGFSVADGLLEKGVYVKNIRPAGPGDLGGLKPYDRLLQVNHVRTRDFDCCLVVPLIAESGNKLDLVISRNPLASQKSIDQQSLPGDWSEQNSAFFQQPSHGGNLETREPTNTL",
# RL36_HUMAN
"MALRYPMAVGLNKGHKVTKNVSKPRHSRRRGRLTKHTKFVRDMIREVCGFAPYERRAMELLKVSKDKRALKFIKKRVGTHIRAKRKREELSNVLAAMRKAAAKKD",
# KIF3A_HUMAN
"MPINKSEKPESCDNVKVVVRCRPLNEREKSMCYKQAVSVDEMRGTITVHKTDSSNEPPKTFTFDTVFGPESKQLDVYNLTARPIIDSVLEGYNGTIFAYGQTGTGKTFTMEGVRAIPELRGIIPNSFAHIFGHIAKAEGDTRFLVRVSYLEIYNEEVRDLLGKDQTQRLEVKERPDVGVYIKDLSAYVVNNADDMDRIMTLGHKNRSVGATNMNEHSSRSHAIFTITIECSEKGIDGNMHVRMGKLHLVDLAGSERQAKTGATGQRLKEATKINLSLSTLGNVISALVDGKSTHVPYRNSKLTRLLQDSLGGNSKTMMCANIGPADYNYDETISTLRYANRAKNIKNKARINEDPKDALLRQFQKEIEELKKKLEEGEEISGSDISGSEEDDDEEGEVGEDGEKRKKRRGKKKVSPDKMIEMQAKIDEERKALETKLDMEEEERNKARAELEKREKDLLKAQQEHQSLLEKLSALEKKVIVGGVDLLAKAEEQEKLLEESNMELEERRKRAEQLRRELEEKEQERLDIEEKYTSLQEEAQGKTKKLKKVWTMLMAAKSEMADLQQEHQREIEGLLENIRQLSRELRLQMLIIDNFIPRDYQEMIENYVHWNEDIGEWQLKCVAYTGNNMRKQTPVPDKKEKDPFEVDLSHVYLAYTEESLRQSLMKLERPRTSKGKARPKTGRRKRSAKPETVIDSLLQ",
# NRX3A_HUMAN
"MSSTLHSVFFTLKVSILLGSLLGLCLGLEFMGLPNQWARYLRWDASTRSDLSFQFKTNVSTGLLLYLDDGGVCDFLCLSLVDGRVQLRFSMDCAETAVLSNKQVNDSSWHFLMVSRDRLRTVLMLDGEGQSGELQPQRPYMDVVSDLFLGGVPTDIRPSALTLDGVQAMPGFKGLILDLKYGNSEPRLLGSRGVQMDAEGPCGERPCENGGICFLLDGHPTCDCSTTGYGGKLCSEDVSQDPGLSHLMMSEQAREENVATFRGSEYLCYDLSQNPIQSSSDEITLSFKTWQRNGLILHTGKSADYVNLALKDGAVSLVINLGSGAFEAIVEPVNGKFNDNAWHDVKVTRNLRQVTISVDGILTTTGYTQEDYTMLGSDDFFYVGGSPSTADLPGSPVSNNFMGCLKEVVYKNNDIRLELSRLARIADTKMKIYGEVVFKCENVATLDPINFETPEAYISLPKWNTKRMGSISFDFRTTEPNGLILFTHGKPQERKDARSQKNTKVDFFAVELLDGNLYLLLDMGSGTIKVKATQKKANDGEWYHVDIQRDGRSGTISVNSRRTPFTASGESEILDLEGDMYLGGLPENRAGLILPTELWTAMLNYGYVGCIRDLFIDGRSKNIRQLAEMQNAAGVKSSCSRMSAKQCDSYPCKNNAVCKDGWNRFICDCTGTGYWGRTCEREASILSYDGSMYMKIIMPMVMHTEAEDVSFRFMSQRAYGLLVATTSRDSADTLRLELDGGRVKLMVNLDCIRINCNSSKGPETLYAGQKLNDNEWHTVRVVRRGKSLKLTVDDDVAEGTMVGDHTRLEFHNIETGIMTEKRYISVVPSSFIGHLQSLMFNGLLYIDLCKNGDIDYCELKARFGLRNIIADPVTFKTKSSYLSLATLQAYTSMHLFFQFKTTSPDGFILFNSGDGNDFIAVELVKGYIHYVFDLGNGPNVIKGNSDRPLNDNQWHNVVITRDNSNTHSLKVDTKVVTQVINGAKNLDLKGDLYMAGLAQGMYSNLPKLVASRDGFQGCLASVDLNGRLPDLINDALHRSGQIERGCEGPSTTCQEDSCANQGVCMQQWEGFTCDCSMTSYSGNQCNDPGATYIFGKSGGLILYTWPANDRPSTRSDRLAVGFSTTVKDGILVRIDSAPGLGDFLQLHIEQGKIGVVFNIGTVDISIKEERTPVNDGKYHVVRFTRNGGNATLQVDNWPVNEHYPTGRQLTIFNTQAQIAIGGKDKGRLFQGQLSGLYYDGLKVLNMAAENNPNIKINGSVRLVGEVPSILGTTQTTSMPPEMSTTVMETTTTMATTTTRKNRSTASIQPTSDDLVSSAECSSDDEDFVECEPSTTGGELVIPLLVEDPLATPPIATRAPSITLPPTFRPLLTIIETTKDSLSMTSEAGLPCLSDQGSDGCDDDGLVISGYGSGETFDSNLPPTDDEDFYTTFSLVTDKSLSTSIFEGGYKAHAPKWESKDFRPNKVSETSRTTTTSLSPELIRFTASSSSGMVPKLPAGKMNNRDLKPQPDIVLLPLPTAYELDSTKLKSPLITSPMFRNVPTANPTEPGIRRVPGASEVIRESSSTTGMVVGIVAAAALCILILLYAMYKYRNRDEGSYQVDETRNYISNSAQSNGTLMKEKQQSSKSGHKKQKNKDREYYV",
# DAAM1_HUMAN
"MAPRKRGGRGISFIFCCFRNNDHPEITYRLRNDSNFALQTMEPALPMPPVEELDVMFSELVDELDLTDKHREAMFALPAEKKWQIYCSKKKDQEENKGATSWPEFYIDQLNSMAARKSLLALEKEEEEERSKTIESLKTALRTKPMRFVTRFIDLDGLSCILNFLKTMDYETSESRIHTSLIGCIKALMNNSQGRAHVLAHSESINVIAQSLSTENIKTKVAVLEILGAVCLVPGGHKKVLQAMLHYQKYASERTRFQTLINDLDKSTGRYRDEVSLKTAIMSFINAVLSQGAGVESLDFRLHLRYEFLMLGIQPVIDKLREHENSTLDRHLDFFEMLRNEDELEFAKRFELVHIDTKSATQMFELTRKRLTHSEAYPHFMSILHHCLQMPYKRSGNTVQYWLLLDRIIQQIVIQNDKGQDPDSTPLENFNIKNVVRMLVNENEVKQWKEQAEKMRKEHNELQQKLEKKERECDAKTQEKEEMMQTLNKMKEKLEKETTEHKQVKQQVADLTAQLHELSRRAVCASIPGGPSPGAPGGPFPSSVPGSLLPPPPPPPLPGGMLPPPPPPLPPGGPPPPPGPPPLGAIMPPPGAPMGLALKKKSIPQPTNALKSFNWSKLPENKLEGTVWTEIDDTKVFKILDLEDLERTFSAYQRQQDFFVNSNSKQKEADAIDDTLSSKLKVKELSVIDGRRAQNCNILLSRLKLSNDEIKRAILTMDEQEDLPKDMLEQLLKFVPEKSDIDLLEEHKHELDRMAKADRFLFEMSRINHYQQRLQSLYFKKKFAERVAEVKPKVEAIRSGSEEVFRSGALKQLLEVVLAFGNYMNKGQRGNAYGFKISSLNKIADTKSSIDKNITLLHYLITIVENKYPSVLNLNEELRDIPQAAKVNMTELDKEISTLRSGLKAVETELEYQKSQPPQPGDKFVSVVSQFITVASFSFSDVEDLLAEAKDLFTKAVKHFGEEAGKIQPDEFFGIFDQFLQAVSEAKQENENMRKKKEEEERRARMEAQLKEQRERERKMRKAKENSEESGEFDDLVSALRSGEVFDKDLSKLKRNRKRITNQMTDSSRERPITKLNF",
# DGLA_HUMAN
"MPGIVVFRRRWSVGSDDLVLPAIFLFLLHTTWFVILSVVLFGLVYNPHEACSLNLVDHGRGYLGILLSCMIAEMAIIWLSMRGGILYTEPRDSMQYVLYVRLAILVIEFIYAIVGIVWLTQYYTSCNDLTAKNVTLGMVVCNWVVILSVCITVLCVFDPTGRTFVKLRATKRRQRNLRTYNLRHRLEEGQATSWSRRLKVFLCCTRTKDSQSDAYSEIAYLFAEFFRDLDIVPSDIIAGLVLLRQRQRAKRNAVLDEANNDILAFLSGMPVTRNTKYLDLKNSQEMLRYKEVCYYMLFALAAYGWPMYLMRKPACGLCQLARSCSCCLCPARPRFAPGVTIEEDNCCGCNAIAIRRHFLDENMTAVDIVYTSCHDAVYETPFYVAVDHDKKKVVISIRGTLSPKDALTDLTGDAERLPVEGHHGTWLGHKGMVLSAEYIKKKLEQEMVLSQAFGRDLGRGTKHYGLIVVGHSLGAGTAAILSFLLRPQYPTLKCFAYSPPGGLLSEDAMEYSKEFVTAVVLGKDLVPRIGLSQLEGFRRQLLDVLQRSTKPKWRIIVGATKCIPKSELPEEVEVTTLASTRLWTHPSDLTIALSASTPLYPPGRIIHVVHNHPAEQCCCCEQEEPTYFAIWGDNKAFNEVIISPAMLHEHLPYVVMEGLNKVLENYNKGKTALLSAAKVMVSPTEVDLTPELIFQQQPLPTGPPMPTGLALELPTADHRNSSVRSKSQSEMSLEGFSEGRLLSPVVAAAARQDPVELLLLSTQERLAAELQARRAPLATMESLSDTESLYSFDSRRSSGFRSIRGSPSLHAVLERDEGHLFYIDPAIPEENPSLSSRTELLAADSLSKHSQDTQPLEAALGSGGVTPERPPSAAANDEEEEVGGGGGGPASRGELALHNGRLGDSPSPQVLEFAEFIDSLFNLDSKSSSFQDLYCMVVPESPTSDYAEGPKSPSQQEILLRAQFEPNLVPKPPRLFAGSADPSSGISLSPSFPLSSSGELMDLTPTGLSSQECLAADKIRTSTPTGHGASPAKQDELVISAR",
# PLXD1_HUMAN
"MAPRAAGGAPLSARAAAASPPPFQTPPRCPVPLLLLLLLGAARAGALEIQRRFPSPTPTNNFALDGAAGTVYLAAVNRLYQLSGANLSLEAEAAVGPVPDSPLCHAPQLPQASCEHPRRLTDNYNKILQLDPGQGLVVVCGSIYQGFCQLRRRGNISAVAVRFPPAAPPAEPVTVFPSMLNVAANHPNASTVGLVLPPAAGAGGSRLLVGATYTGYGSSFFPRNRSLEDHRFENTPEIAIRSLDTRGDLAKLFTFDLNPSDDNILKIKQGAKEQHKLGFVSAFLHPSDPPPGAQSYAYLALNSEARAGDKESQARSLLARICLPHGAGGDAKKLTESYIQLGLQCAGGAGRGDLYSRLVSVFPARERLFAVFERPQGSPAARAAPAALCAFRFADVRAAIRAARTACFVEPAPDVVAVLDSVVQGTGPACERKLNIQLQPEQLDCGAAHLQHPLSILQPLKATPVFRAPGLTSVAVASVNNYTAVFLGTVNGRLLKINLNESMQVVSRRVVTVAYGEPVHHVMQFDPADSGYLYLMTSHQMARVKVAACNVHSTCGDCVGAADAYCGWCALETRCTLQQDCTNSSQQHFWTSASEGPSRCPAMTVLPSEIDVRQEYPGMILQISGSLPSLSGMEMACDYGNNIRTVARVPGPAFGHQIAYCNLLPRDQFPPFPPNQDHVTVEMSVRVNGRNIVKANFTIYDCSRTAQVYPHTACTSCLSAQWPCFWCSQQHSCVSNQSRCEASPNPTSPQDCPRTLLSPLAPVPTGGSQNILVPLANTAFFQGAALECSFGLEEIFEAVWVNESVVRCDQVVLHTTRKSQVFPLSLQLKGRPARFLDSPEPMTVMVYNCAMGSPDCSQCLGREDLGHLCMWSDGCRLRGPLQPMAGTCPAPEIHAIEPLSGPLDGGTLLTIRGRNLGRRLSDVAHGVWIGGVACEPLPDRYTVSEEIVCVTGPAPGPLSGVVTVNASKEGKSRDRFSYVLPLVHSLEPTMGPKAGGTRITIHGNDLHVGSELQVLVNDTDPCTELMRTDTSIACTMPEGALPAPVPVCVRFERRGCVHGNLTFWYMQNPVITAISPRRSPVSGGRTITVAGERFHMVQNVSMAVHHIGREPTLCKVLNSTLITCPSPGALSNASAPVDFFINGRAYADEVAVAEELLDPEEAQRGSRFRLDYLPNPQFSTAKREKWIKHHPGEPLTLVIHKEQDSLGLQSHEYRVKIGQVSCDIQIVSDRIIHCSVNESLGAAVGQLPITIQVGNFNQTIATLQLGGSETAIIVSIVICSVLLLLSVVALFVFCTKSRRAERYWQKTLLQMEEMESQIREEIRKGFAELQTDMTDLTKELNRSQGIPFLEYKHFVTRTFFPKCSSLYEERYVLPSQTLNSQGSSQAQETHPLLGEWKIPESCRPNMEEGISLFSSLLNNKHFLIVFVHALEQQKDFAVRDRCSLASLLTIALHGKLEYYTSIMKELLVDLIDASAAKNPKLMLRRTESVVEKMLTNWMSICMYSCLRETVGEPFFLLLCAIKQQINKGSIDAITGKARYTLSEEWLLRENIEAKPRNLNVSFQGCGMDSLSVRAMDTDTLTQVKEKILEAFCKNVPYSQWPRAEDVDLEWFASSTQSYILRDLDDTSVVEDGRKKLNTLAHYKIPEGASLAMSLIDKKDNTLGRVKDLDTEKYFHLVLPTDELAEPKKSHRQSHRKKVLPEIYLTRLLSTKGTLQKFLDDLFKAILSIREDKPPLAVKYFFDFLEEQAEKRGISDPDTLHIWKTNSLPLRFWVNILKNPQFVFDIDKTDHIDACLSVIAQAFIDACSISDLQLGKDSPTNKLLYAKEIPEYRKIVQRYYKQIQDMTPLSEQEMNAHLAEESRKYQNEFNTNVAMAEIYKYAKRYRPQIMAALEANPTARRTQLQHKFEQVVALMEDNIYECYSEA",
# WDR7_HUMAN
"MAGNSLVLPIVLWGRKAPTHCISAVLLTDDGATIVTGCHDGQICLWDLSVELQINPRALLFGHTASITCLSKACASSDKQYIVSASESGEMCLWDVSDGRCIEFTKLACTHTGIQFYQFSVGNQREGRLLCHGHYPEILVVDATSLEVLYSLVSKISPDWISSMSIIRSHRTQEDTVVALSVTGILKVWIVTSEISDMQDTEPIFEEESKPIYCQNCQSISFCAFTQRSLLVVCSKYWRVFDAGDYSLLCSGPSENGQTWTGGDFVSSDKVIIWTENGQSYIYKLPASCLPASDSFRSDVGKAVENLIPPVQHILLDRKDKELLICPPVTRFFYGCREYFHKLLIQGDSSGRLNIWNISDTADKQGSEEGLAMTTSISLQEAFDKLNPCPAGIIDQLSVIPNSNEPLKVTASVYIPAHGRLVCGREDGSIVIVPATQTAIVQLLQGEHMLRRGWPPHRTLRGHRNKVTCLLYPHQVSARYDQRYLISGGVDFSVIIWDIFSGEMKHIFCVHGGEITQLLVPPENCSARVQHCICSVASDHSVGLLSLREKKCIMLASRHLFPIQVIKWRPSDDYLVVGCSDGSVYVWQMDTGALDRCVMGITAVEILNACDEAVPAAVDSLSHPAVNLKQAMTRRSLAALKNMAHHKLQTLATNLLASEASDKGNLPKYSHNSLMVQAIKTNLTDPDIHVLFFDVEALIIQLLTEEASRPNTALISPENLQKASGSSDKGGSFLTGKRAAVLFQQVKETIKENIKEHLLDDEEEDEEIMRQRREESDPEYRSSKSKPLTLLEYNLTMDTAKLFMSCLHAWGLNEVLDEVCLDRLGMLKPHCTVSFGLLSRGGHMSLMLPGYNQPACKLSHGKTEVGRKLPASEGVGKGTYGVSRAVTTQHLLSIISLANTLMSMTNATFIGDHMKKGPTRPPRPSTPDLSKARGSPPTSSNIVQGQIKQVAAPVVSARSDADHSGSDPPSAPALHTCFLVNEGWSQLAAMHCVMLPDLLGLDKFRPPLLEMLARRWQDRCLEVREAAQALLLAELRRIEQAGRKEAIDAWAPYLPQYIDHVISPGVTSEAAQTITTAPDASGPEAKVQEEEHDLVDDDITTGCLSSVPQMKKISTSYEERRKQATAIVLLGVIGAEFGAEIEPPKLLTRPRSSSQIPEGFGLTSGGSNYSLARHTCKALTFLLLQPPSPKLPPHSTIRRTAIDLIGRGFTVWEPYMDVSAVLMGLLELCADAEKQLANITMGLPLSPAADSARSARHALSLIATARPPAFITTIAKEVHRHTALAANTQSQQNMHTTTLARAKGEILRVIEILIEKMPTDVVDLLVEVMDIIMYCLEGSLVKKKGLQECFPAICRFYMVSYYERNHRIAVGARHGSVALYDIRTGKCQTIHGHKGPITAVAFAPDGRYLATYSNTDSHISFWQMNTSLLGSIGMLNSAPQLRCIKTYQVPPVQPASPGSHNALKLARLIWTSNRNVILMAHDGKEHRFMV",
# FARP1_HUMAN
"MGEIEQRPTPGSRLGAPENSGISTLERGQKPPPTPSGKLVSIKIQMLDDTQEAFEVPQRAPGKVLLDAVCNHLNLVEGDYFGLEFPDHKKITVWLDLLKPIVKQIRRPKHVVVKFVVKFFPPDHTQLQEELTRYLFALQVKQDLAQGRLTCNDTSAALLISHIVQSEIGDFDEALDREHLAKNKYIPQQDALEDKIVEFHHNHIGQTPAESDFQLLEIARRLEMYGIRLHPAKDREGTKINLAVANTGILVFQGFTKINAFNWAKVRKLSFKRKRFLIKLRPDANSAYQDTLEFLMASRDFCKSFWKICVEHHAFFRLFEEPKPKPKPVLFSRGSSFRFSGRTQKQVLDYVKEGGHKKVQFERKHSKIHSIRSLASQPTELNSEVLEQSQQSTSLTFGEGAESPGGQSCRRGKEPKVSAGEPGSHPSPAPRRSPAGNKQADGAASAPTEEEEEVVKDRTQQSKPQPPQPSTGSLTGSPHLSELSVNSQGGVAPANVTLSPNLSPDTKQASPLISPLLNDQACPRTDDEDEGRRKRFPTDKAYFIAKEVSTTERTYLKDLEVITSWFQSTVSKEDAMPEALKSLIFPNFEPLHKFHTNFLKEIEQRLALWEGRSNAQIRDYQRIGDVMLKNIQGMKHLAAHLWKHSEALEALENGIKSSRRLENFCRDFELQKVCYLPLNTFLLRPLHRLMHYKQVLERLCKHHPPSHADFRDCRAALAEITEMVAQLHGTMIKMENFQKLHELKKDLIGIDNLVVPGREFIRLGSLSKLSGKGLQQRMFFLFNDVLLYTSRGLTASNQFKVHGQLPLYGMTIEESEDEWGVPHCLTLRGQRQSIIVAASSRSEMEKWVEDIQMAIDLAEKSSSPAPEFLASSPPDNKSPDEATAADQESEDDLSASRTSLERQAPHRGNTMVHVCWHRNTSVSMVDFSIAVENQLSGNLLRKFKNSNGWQKLWVVFTNFCLFFYKSHQDNHPLASLPLLGYSLTIPSESENIQKDYVFKLHFKSHVYYFRAESEYTFERWMEVIRSATSSASRPHVLSHKESLVY",
# TLN2_HUMAN
"MVALSLKICVRHCNVVKTMQFEPSTAVYDACRVIRERVPEAQTGQASDYGLFLSDEDPRKGIWLEAGRTLDYYMLRNGDILEYKKKQRPQKIRMLDGSVKTVMVDDSKTVGELLVTICSRIGITNYEEYSLIQETIEEKKEEGTGTLKKDRTLLRDERKMEKLKAKLHTDDDLNWLDHSRTFREQGVDENETLLLRRKFFYSDQNVDSRDPVQLNLLYVQARDDILNGSHPVSFEKACEFGGFQAQIQFGPHVEHKHKPGFLDLKEFLPKEYIKQRGAEKRIFQEHKNCGEMSEIEAKVKYVKLARSLRTYGVSFFLVKEKMKGKNKLVPRLLGITKDSVMRVDEKTKEVLQEWPLTTVKRWAASPKSFTLDFGEYQESYYSVQTTEGEQISQLIAGYIDIILKKKQSKDRFGLEGDEESTMLEESVSPKKSTILQQQFNRTGKAEHGSVALPAVMRSGSSGPETFNVGSMPSPQQQVMVGQMHRGHMPPLTSAQQALMGTINTSMHAVQQAQDDLSELDSLPPLGQDMASRVWVQNKVDESKHEIHSQVDAITAGTASVVNLTAGDPADTDYTAVGCAITTISSNLTEMSKGVKLLAALMDDEVGSGEDLLRAARTLAGAVSDLLKAVQPTSGEPRQTVLTAAGSIGQASGDLLRQIGENETDERFQDVLMSLAKAVANAAAMLVLKAKNVAQVAEDTVLQNRVIAAATQCALSTSQLVACAKVVSPTISSPVCQEQLIEAGKLVDRSVENCVRACQAATTDSELLKQVSAAASVVSQALHDLLQHVRQFASRGEPIGRYDQATDTIMCVTESIFSSMGDAGEMVRQARVLAQATSDLVNAMRSDAEAEIDMENSKKLLAAAKLLADSTARMVEAAKGAAANPENEDQQQRLREAAEGLRVATNAAAQNAIKKKIVNRLEVAAKQAAAAATQTIAASQNAAVSNKNPAAQQQLVQSCKAVADHIPQLVQGVRGSQAQAEDLSAQLALIISSQNFLQPGSKMVSSAKAAVPTVSDQAAAMQLSQCAKNLATSLAELRTASQKAHEACGPMEIDSALNTVQTLKNELQDAKMAAVESQLKPLPGETLEKCAQDLGSTSKAVGSSMAQLLTCAAQGNEHYTGVAARETAQALKTLAQAARGVAASTTDPAAAHAMLDSARDVMEGSAMLIQEAKQALIAPGDAERQQRLAQVAKAVSHSLNNCVNCLPGQKDVDVALKSIGESSKKLLVDSLPPSTKPFQEAQSELNQAAADLNQSAGEVVHATRGQSGELAAASGKFSDDFDEFLDAGIEMAGQAQTKEDQIQVIGNLKNISMASSKLLLAAKSLSVDPGAPNAKNLLAAAARAVTESINQLITLCTQQAPGQKECDNALRELETVKGMLDNPNEPVSDLSYFDCIESVMENSKVLGESMAGISQNAKTGDLPAFGECVGIASKALCGLTEAAAQAAYLVGISDPNSQAGHQGLVDPIQFARANQAIQMACQNLVDPGSSPSQVLSAATIVAKHTSALCNACRIASSKTANPVAKRHFVQSAKEVANSTANLVKTIKALDGDFSEDNRNKCRIATAPLIEAVENLTAFASNPEFVSIPAQISSEGSQAQEPILVSAKTMLESSSYLIRTARSLAINPKDPPTWSVLAGHSHTVSDSIKSLITSIRDKAPGQRECDYSIDGINRCIRDIEQASLAAVSQSLATRDDISVEALQEQLTSVVQEIGHLIDPIATAARGEAAQLGHKVTQLASYFEPLILAAVGVASKILDHQQQMTVLDQTKTLAESALQMLYAAKEGGGNPKAQHTHDAITEAAQLMKEAVDDIMVTLNEAASEVGLVGGMVDAIAEAMSKLDEGTPPEPKGTFVDYQTTVVKYSKAIAVTAQEMMTKSVTNPEELGGLASQMTSDYGHLAFQGQMAAATAEPEEIGFQIRTRVQDLGHGCIFLVQKAGALQVCPTDSYTKRELIECARAVTEKVSLVLSALQAGNKGTQACITAATAVSGIIADLDTTIMFATAGTLNAENSETFADHRENILKTAKALVEDTKLLVSGAASTPDKLAQAAQSSAATITQLAEVVKLGAASLGSDDPETQVVLINAIKDVAKALSDLISATKGAASKPVDDPSMYQLKGAAKVMVTNVTSLLKTVKAVEDEATRGTRALEATIECIKQELTVFQSKDVPEKTSSPEESIRMTKGITMATAKAVAAGNSCRQEDVIATANLSRKAVSDMLTACKQASFHPDVSDEVRTRALRFGTECTLGYLDLLEHVLVILQKPTPEFKQQLAAFSKRVAGAVTELIQAAEAMKGTEWVDPEDPTVIAETELLGAAASIEAAAKKLEQLKPRAKPKQADETLDFEEQILEAAKSIAAATSALVKSASAAQRELVAQGKVGSIPANAADDGQWSQGLISAARMVAAATSSLCEAANASVQGHASEEKLISSAKQVAASTAQLLVACKVKADQDSEAMRRLQAAGNAVKRASDNLVRAAQKAAFGKADDDDVVVKTKFVGGIAQIIAAQEEMLKKERELEEARKKLAQIRQQQYKFLPTELREDEG",
# RPGF2_HUMAN
"MKPLAIPANHGVMGQQEKHSLPADFTKLHLTDSLHPQVTHVSSSHSGCSITSDSGSSSLSDIYQATESEAGDMDLSGLPETAVDSEDDDDEEDIERASDPLMSRDIVRDCLEKDPIDRTDDDIEQLLEFMHQLPAFANMTMSVRRELCAVMVFAVVERAGTIVLNDGEELDSWSVILNGSVEVTYPDGKAEILCMGNSFGVSPTMDKEYMKGVMRTKVDDCQFVCIAQQDYCRILNQVEKNMQKVEEEGEIVMVKEHRELDRTGTRKGHIVIKGTSERLTMHLVEEHSVVDPTFIEDFLLTYRTFLSSPMEVGKKLLEWFNDPSLRDKVTRVVLLWVNNHFNDFEGDPAMTRFLEEFENNLEREKMGGHLRLLNIACAAKAKRRLMTLTKPSREAPLPFILLGGSEKGFGIFVDSVDSGSKATEAGLKRGDQILEVNGQNFENIQLSKAMEILRNNTHLSITVKTNLFVFKELLTRLSEEKRNGAPHLPKIGDIKKASRYSIPDLAVDVEQVIGLEKVNKKSKANTVGGRNKLKKILDKTRISILPQKPYNDIGIGQSQDDSIVGLRQTKHIPTALPVSGTLSSSNPDLLQSHHRILDFSATPDLPDQVLRVFKADQQSRYIMISKDTTAKEVVIQAIREFAVTATPDQYSLCEVSVTPEGVIKQRRLPDQLSKLADRIQLSGRYYLKNNMETETLCSDEDAQELLRESQISLLQLSTVEVATQLSMRNFELFRNIEPTEYIDDLFKLRSKTSCANLKRFEEVINQETFWVASEILRETNQLKRMKIIKHFIKIALHCRECKNFNSMFAIISGLNLAPVARLRTTWEKLPNKYEKLFQDLQDLFDPSRNMAKYRNVLNSQNLQPPIIPLFPVIKKDLTFLHEGNDSKVDGLVNFEKLRMIAKEIRHVGRMASVNMDPALMFRTRKKKWRSLGSLSQGSTNATVLDVAQTGGHKKRVRRSSFLNAKKLYEDAQMARKVKQYLSNLELEMDEESLQTLSLQCEPATNTLPKNPGDKKPVKSETSPVAPRAGSQQKAQSLPQPQQQPPPAHKINQGLQVPAVSLYPSRKKVPVKDLPPFGINSPQALKKILSLSEEGSLERHKKQAEDTISNASSQLSSPPTSPQSSPRKGYTLAPSGTVDNFSDSGHSEISSRSSIVSNSSFDSVPVSLHDERRQRHSVSIVETNLGMGRMERRTMIEPDQYSLGSYAPMSEGRGLYATATVISSPSTEELSQDQGDRASLDAADSGRGSWTSCSSGSHDNIQTIQHQRSWETLPFGHTHFDYSGDPAGLWASSSHMDQIMFSDHSTKYNRQNQSRESLEQAQSRASWASSTGYWGEDSEGDTGTIKRRGGKDVSIEAESSSLTSVTTEETKPVPMPAHIAVASSTTKGLIARKEGRYREPPPTPPGYIGIPITDFPEGHSHPARKPPDYNVALQRSRMVARSSDTAGPSSVQQPHGHPTSSRPVNKPQWHKPNESDPRLAPYQSQGFSTEEDEDEQVSAV",
# MYO5A_HUMAN
"MAASELYTKFARVWIPDPEEVWKSAELLKDYKPGDKVLLLHLEEGKDLEYHLDPKTKELPHLRNPDILVGENDLTALSYLHEPAVLHNLRVRFIDSKLIYTYCGIVLVAINPYEQLPIYGEDIINAYSGQNMGDMDPHIFAVAEEAYKQMARDERNQSIIVSGESGAGKTVSAKYAMRYFATVSGSASEANVEEKVLASNPIMESIGNAKTTRNDNSSRFGKYIEIGFDKRYRIIGANMRTYLLEKSRVVFQAEEERNYHIFYQLCASAKLPEFKMLRLGNADNFNYTKQGGSPVIEGVDDAKEMAHTRQACTLLGISESHQMGIFRILAGILHLGNVGFTSRDADSCTIPPKHEPLCIFCELMGVDYEEMCHWLCHRKLATATETYIKPISKLQATNARDALAKHIYAKLFNWIVDNVNQALHSAVKQHSFIGVLDIYGFETFEINSFEQFCINYANEKLQQQFNMHVFKLEQEEYMKEQIPWTLIDFYDNQPCINLIESKLGILDLLDEECKMPKGTDDTWAQKLYNTHLNKCALFEKPRLSNKAFIIQHFADKVEYQCEGFLEKNKDTVFEEQIKVLKSSKFKMLPELFQDDEKAISPTSATSSGRTPLTRTPAKPTKGRPGQMAKEHKKTVGHQFRNSLHLLMETLNATTPHYVRCIKPNDFKFPFTFDEKRAVQQLRACGVLETIRISAAGFPSRWTYQEFFSRYRVLMKQKDVLSDRKQTCKNVLEKLILDKDKYQFGKTKIFFRAGQVAYLEKLRADKLRAACIRIQKTIRGWLLRKKYLRMRKAAITMQRYVRGYQARCYAKFLRRTKAATIIQKYWRMYVVRRRYKIRRAATIVLQSYLRGFLARNRYRKILREHKAVIIQKRVRGWLARTHYKRSMHAIIYLQCCFRRMMAKRELKKLKIEARSVERYKKLHIGMENKIMQLQRKVDEQNKDYKCLVEKLTNLEGIYNSETEKLRSDLERLQLSEEEAKVATGRVLSLQEEIAKLRKDLEQTRSEKKCIEEHADRYKQETEQLVSNLKEENTLLKQEKEALNHRIVQQAKEMTETMEKKLVEETKQLELDLNDERLRYQNLLNEFSRLEERYDDLKEEMTLMVHVPKPGHKRTDSTHSSNESEYIFSSEIAEMEDIPSRTEEPSEKKVPLDMSLFLKLQKRVTELEQEKQVMQDELDRKEEQVLRSKAKEEERPQIRGAELEYESLKRQELESENKKLKNELNELRKALSEKSAPEVTAPGAPAYRVLMEQLTSVSEELDVRKEEVLILRSQLVSQKEAIQPKDDKNTMTDSTILLEDVQKMKDKGEIAQAYIGLKETNRSSALDYHELNEDGELWLVYEGLKQANRLLESQLQSQKRSHENEAEALRGEIQSLKEENNRQQQLLAQNLQLPPEARIEASLQHEITRLTNENLDLMEQLEKQDKTVRKLKKQLKVFAKKIGELEVGQMENISPGQIIDEPIRPVNIPRKEKDFQGMLEYKKEDEQKLVKNLILELKPRGVAVNLIPGLPAYILFMCVRHADYLNDDQKVRSLLTSTINSIKKVLKKRGDDFETVSFWLSNTCRFLHCLKQYSGEEGFMKHNTSRQNEHCLTNFDLAEYRQVLSDLAIQIYQQLVRVLENILQPMIVSGMLEHETIQGVSGVKPTGLRKRTSSIADEGTYTLDSILRQLNSFHSVMCQHGMDPELIKQVVKQMFYIIGAITLNNLLLRKDMCSWSKGMQIRYNVSQLEEWLRDKNLMNSGAKETLEPLIQAAQLLQVKKKTDDDAEAICSMCNALTTAQIVKVLNLYTPVNEFEERVSVSFIRTIQMRLRDRKDSPQLLMDAKHIFPVTFPFNPSSLALETIQIPASLGLGFISRV",
# TRAF6_HUMAN
"MSLLNCENSCGSSQSESDCCVAMASSCSAVTKDDSVGGTASTGNLSSSFMEEIQGYDVEFDPPLESKYECPICLMALREAVQTPCGHRFCKACIIKSIRDAGHKCPVDNEILLENQLFPDNFAKREILSLMVKCPNEGCLHKMELRHLEDHQAHCEFALMDCPQCQRPFQKFHINIHILKDCPRRQVSCDNCAASMAFEDKEIHDQNCPLANVICEYCNTILIREQMPNHYDLDCPTAPIPCTFSTFGCHEKMQRNHLARHLQENTQSHMRMLAQAVHSLSVIPDSGYISEVRNFQETIHQLEGRLVRQDHQIRELTAKMETQSMYVSELKRTIRTLEDKVAEIEAQQCNGIYIWKIGNFGMHLKCQEEEKPVVIHSPGFYTGKPGYKLCMRLHLQLPTAQRCANYISLFVHTMQGEYDSHLPWPFQGTIRLTILDQSEAPVRQNHEEIMDAKPELLAFQRPTIPRNPKGFGYVTFMHLEALRQRTFIKDDTLLVRCEVSTRFDMGSLRREGFQPRSTDAGV",
# SHAN1_HUMAN
"MTHSPATSEDEERHSASECPEGGSESDSSPDGPGRGPRGTRGQGSGAPGSLASVRGLQGRSMSVPDDAHFSMMVFRIGIPDLHQTKCLRFNPDATIWTAKQQVLCALSESLQDVLNYGLFQPATSGRDANFLEEERLLREYPQSFEKGVPYLEFRYKTRVYKQTNLDEKQLAKLHTKTGLKKFLEYVQLGTSDKVARLLDKGLDPNYHDSDSGETPLTLAAQTEGSVEVIRTLCLGGAHIDFRARDGMTALHKAACARHCLALTALLDLGGSPNYKDRRGLTPLFHTAMVGGDPRCCELLLFNRAQLGIADENGWQEIHQACQRGHSQHLEHLLFYGAEPGAQNASGNTALHICALYNKETCARILLYRGADKDVKNNNGQTPFQVAVIAGNFELGELIRNHREQDVVPFQESPKYAARRRGPPGTGLTVPPALLRANSDTSMALPDWMVFSAPGAASSGAPGPTSGSQGQSQPSAPTTKLSSGTLRSASSPRGARARSPSRGRHPEDAKRQPRGRPSSSGTPREGPAGGTGGSGGPGGSLGSRGRRRKLYSAVPGRSFMAVKSYQAQAEGEISLSKGEKIKVLSIGEGGFWEGQVKGRVGWFPSDCLEEVANRSQESKQESRSDKAKRLFRHYTVGSYDSFDAPSLMDGIGPGSDYIIKEKTVLLQKKDSEGFGFVLRGAKAQTPIEEFTPTPAFPALQYLESVDEGGVAWRAGLRMGDFLIEVNGQNVVKVGHRQVVNMIRQGGNTLMVKVVMVTRHPDMDEAVHKKAPQQAKRLPPPTISLRSKSMTSELEEMEYEQQPAPVPSMEKKRTVYQMALNKLDEILAAAQQTISASESPGPGGLASLGKHRPKGFFATESSFDPHHRAQPSYERPSFLPPGPGLMLRQKSIGAAEDDRPYLAPPAMKFSRSLSVPGSEDIPPPPTTSPPEPPYSTPPVPSSSGRLTPSPRGGPFNPGSGGPLPASSPASFDGPSPPDTRVGSREKSLYHSGPLPPAHHHPPHHHHHHAPPPQPHHHHAHPPHPPEMETGGSPDDPPPRLALGPQPSLRGWRGGGPSPTPGAPSPSHHGSAGGGGGSSQGPALRYFQLPPRAASAAMYVPARSGRGRKGPLVKQTKVEGEPQKGGGLPPAPSPTSPASPQPPPAVAAPSEKNSIPIPTIIIKAPSTSSSGRSSQGSSTEAEPPTQPEPTGGGGGGGSSPSPAPAMSPVPPSPSPVPTPASPSGPATLDFTSQFGAALVGAARREGGWQNEARRRSTLFLSTDAGDEDGGDGGLGTGAAPGPRLRHSKSIDEGMFSAEPYLRLESAGSGAGYGGYGAGSRAYGGGGGSSAFTSFLPPRPLVHPLTGKALDPASPLGLALAARERALKESSEGGGAPQPPPRPPSPRYEAPPPTPHHHSPHAHHEPVLRLWGASPPDPARRELGYRAGLGSQEKSLPASPPAARRSLLHRLPPTAPGVGPLLLQLGTEPPAPHPGVSKPWRSAAPEEPERLPLHVRFLENCQPRAPVTSGRGPPSEDGPGVPPPSPRRSVPPSPTSPRASEENGLPLLVLPPPAPSVDVEDGEFLFVEPLPPPLEFSNSFEKPESPLTPGPPHPLPDTPAPATPLPPVPPPAVAAAPPTLDSTASSLTSYDSEVATLTQGASAAPGDPHPPGPPAPAAPAPAAPQPGPDPPPGTDSGIEEVDSRSSSDHPLETISSASTLSSLSAEGGGSAGGGGGAGAGVASGPELLDTYVAYLDGQAFGGSSTPGPPYPPQLMTPSKLRGRALGASGGLRPGPSGGLRDPVTPTSPTVSVTGAGTDGLLALRACSGPPTAGVAGGPVAVEPEVPPVPLPTASSLPRKLLPWEEGPGPPPPPLPGPLAQPQASALATVKASIISELSSKLQQFGGSSAAGGALPWARGGSGGGGDSHHGGASYVPERTSSLQRQRLSDDSQSSLLSKPVSSLFQNWPKPPLPPLPTGTGVSPTAAAAPGATSPSASSSSTSTRHLQGVEFEMRPPLLRRAPSPSLLPASEHKVSPAPRPSSLPILPSGPLYPGLFDIRGSPTGGAGGSADPFAPVFVPPHPGISGGLGGALSGASRSLSPTRLLSLPPDKPFGAKPLGFWTKFDVADWLEWLGLAEHRAQFLDHEIDGSHLPALTKEDYVDLGVTRVGHRMNIDRALKFFLER",
# CD2AP_HUMAN
"MVDYIVEYDYDAVHDDELTIRVGEIIRNVKKLQEEGWLEGELNGRRGMFPDNFVKEIKRETEFKDDSLPIKRERHGNVASLVQRISTYGLPAGGIQPHPQTKNIKKKTKKRQCKVLFEYIPQNEDELELKVGDIIDINEEVEEGWWSGTLNNKLGLFPSNFVKELEVTDDGETHEAQDDSETVLAGPTSPIPSLGNVSETASGSVTQPKKIRGIGFGDIFKEGSVKLRTRTSSSETEEKKPEKPLILQSLGPKTQSVEITKTDTEGKIKAKEYCRTLFAYEGTNEDELTFKEGEIIHLISKETGEAGWWRGELNGKEGVFPDNFAVQINELDKDFPKPKKPPPPAKAPAPKPELIAAEKKYFSLKPEEKDEKSTLEQKPSKPAAPQVPPKKPTPPTKASNLLRSSGTVYPKRPEKPVPPPPPIAKINGEVSSISSKFETEPVSKLKLDSEQLPLRPKSVDFDSLTVRTSKETDVVNFDDIASSENLLHLTANRPKMPGRRLPGRFNGGHSPTHSPEKILKLPKEEDSANLKPSELKKDTCYSPKPSVYLSTPSSASKANTTAFLTPLEIKAKVETDDVKKNSLDELRAQIIELLCIVEALKKDHGKELEKLRKDLEEEKTMRSNLEMEIEKLKKAVLSS",
# VATD_HUMAN
"MSGKDRIEIFPSRMAQTIMKARLKGAQTGRNLLKKKSDALTLRFRQILKKIIETKMLMGEVMREAAFSLAEAKFTAGDFSTTVIQNVNKAQVKIRAKKDNVAGVTLPVFEHYHEGTDSYELTGLARGGEQLAKLKRNYAKAVELLVELASLQTSFVTLDEAIKITNRRVNAIEHVIIPRIERTLAYIITELDEREREEFYRLKKIQEKKKILKEKSEKDLEQRRAAGEVLEPANLLAEEKDEDLLFE",
# SNX14_HUMAN
"MVPWVRTMGQKLKQRLRLDVGREICRQYPLFCFLLLCLSAASLLLNRYIHILMIFWSFVAGVVTFYCSLGPDSLLPNIFFTIKYKPKQLGLQELFPQGHSCAVCGKVKCKRHRPSLLLENYQPWLDLKISSKVDASLSEVLELVLENFVYPWYRDVTDDESFVDELRITLRFFASVLIRRIHKVDIPSIITKKLLKAAMKHIEVIVKARQKVKNTEFLQQAALEEYGPELHVALRSRRDELHYLRKLTELLFPYILPPKATDCRSLTLLIREILSGSVFLPSLDFLADPDTVNHLLIIFIDDSPPEKATEPASPLVPFLQKFAEPRNKKPSVLKLELKQIREQQDLLFRFMNFLKQEGAVHVLQFCLTVEEFNDRILRPELSNDEMLSLHEELQKIYKTYCLDESIDKIRFDPFIVEEIQRIAEGPYIDVVKLQTMRCLFEAYEHVLSLLENVFTPMFCHSDEYFRQLLRGAESPTRNSKLNRGSLSLDDFRNTQKRGESFGISRIGSKIKGVFKSTTMEGAMLPNYGVAEGEDDFIEEGIVVMEDDSPVEAVSTPNTPRNLAAWKISIPYVDFFEDPSSERKEKKERIPVFCIDVERNDRRAVGHEPEHWSVYRRYLEFYVLESKLTEFHGAFPDAQLPSKRIIGPKNYEFLKSKREEFQEYLQKLLQHPELSNSQLLADFLSPNGGETQFLDKILPDVNLGKIIKSVPGKLMKEKGQHLEPFIMNFINSCESPKPKPSRPELTILSPTSENNKKLFNDLFKNNANRAENTERKQNQNYFMEVMTVEGVYDYLMYVGRVVFQVPDWLHHLLMGTRILFKNTLEMYTDYYLQCKLEQLFQEHRLVSLITLLRDAIFCENTEPRSLQDKQKGAKQTFEEMMNYIPDLLVKCIGEETKYESIRLLFDGLQQPVLNKQLTYVLLDIVIQELFPELNKVQKEVTSVTSWM",
# SNX9_HUMAN
"MATKARVMYDFAAEPGNNELTVNEGEIITITNPDVGGGWLEGRNIKGERGLVPTDYVEILPSDGKDQFSCGNSVADQAFLDSLSASTAQASSSAASNNHQVGSGNDPWSAWSASKSGNWESSEGWGAQPEGAGAQRNTNTPNNWDTAFGHPQAYQGPATGDDDDWDEDWDGPKSSSYFKDSESADAGGAQRGNSRASSSSMKIPLNKFPGFAKPGTEQYLLAKQLAKPKEKIPIIVGDYGPMWVYPTSTFDCVVADPRKGSKMYGLKSYIEYQLTPTNTNRSVNHRYKHFDWLYERLLVKFGSAIPIPSLPDKQVTGRFEEEFIKMRMERLQAWMTRMCRHPVISESEVFQQFLNFRDEKEWKTGKRKAERDELAGVMIFSTMEPEAPDLDLVEIEQKCEAVGKFTKAMDDGVKELLTVGQEHWKRCTGPLPKEYQKIGKALQSLATVFSSSGYQGETDLNDAITEAGKTYEEIASLVAEQPKKDLHFLMECNHEYKGFLGCFPDIIGTHKGAIEKVKESDKLVATSKITLQDKQNMVKRVSIMSYALQAEMNHFHSNRIYDYNSVIRLYLEQQVQFYETIAEKLRQALSRFPVM",
# SCNAA_HUMAN
"MEFPIGSLETNNFRRFTPESLVEIEKQIAAKQGTKKAREKHREQKDQEEKPRPQLDLKACNQLPKFYGELPAELIGEPLEDLDPFYSTHRTFMVLNKGRTISRFSATRALWLFSPFNLIRRTAIKVSVHSWFSLFITVTILVNCVCMTRTDLPEKIEYVFTVIYTFEALIKILARGFCLNEFTYLRDPWNWLDFSVITLAYVGTAIDLRGISGLRTFRVLRALKTVSVIPGLKVIVGALIHSVKKLADVTILTIFCLSVFALVGLQLFKGNLKNKCVKNDMAVNETTNYSSHRKPDIYINKRGTSDPLLCGNGSDSGHCPDGYICLKTSDNPDFNYTSFDSFAWAFLSLFRLMTQDSWERLYQQTLRTSGKIYMIFFVLVIFLGSFYLVNLILAVVTMAYEEQNQATTDEIEAKEKKFQEALEMLRKEQEVLAALGIDTTSLHSHNGSPLTSKNASERRHRIKPRVSEGSTEDNKSPRSDPYNQRRMSFLGLASGKRRASHGSVFHFRSPGRDISLPEGVTDDGVFPGDHESHRGSLLLGGGAGQQGPLPRSPLPQPSNPDSRHGEDEHQPPPTSELAPGAVDVSAFDAGQKKTFLSAEYLDEPFRAQRAMSVVSIITSVLEELEESEQKCPPCLTSLSQKYLIWDCCPMWVKLKTILFGLVTDPFAELTITLCIVVNTIFMAMEHHGMSPTFEAMLQIGNIVFTIFFTAEMVFKIIAFDPYYYFQKKWNIFDCIIVTVSLLELGVAKKGSLSVLRSFRLLRVFKLAKSWPTLNTLIKIIGNSVGALGNLTIILAIIVFVFALVGKQLLGENYRNNRKNISAPHEDWPRWHMHDFFHSFLIVFRILCGEWIENMWACMEVGQKSICLILFLTVMVLGNLVVLNLFIALLLNSFSADNLTAPEDDGEVNNLQVALARIQVFGHRTKQALCSFFSRSCPFPQPKAEPELVVKLPLSSSKAENHIAANTARGSSGGLQAPRGPRDEHSDFIANPTVWVSVPIAEGESDLDDLEDDGGEDAQSFQQEVIPKGQQEQLQQVERCGDHLTPRSPGTGTSSEDLAPSLGETWKDESVPQVPAEGVDDTSSSEGSTVDCLDPEEILRKIPELADDLEEPDDCFTEGCIRHCPCCKLDTTKSPWDVGWQVRKTCYRIVEHSWFESFIIFMILLSSGSLAFEDYYLDQKPTVKALLEYTDRVFTFIFVFEMLLKWVAYGFKKYFTNAWCWLDFLIVNISLISLTAKILEYSEVAPIKALRTLRALRPLRALSRFEGMRVVVDALVGAIPSIMNVLLVCLIFWLIFSIMGVNLFAGKFWRCINYTDGEFSLVPLSIVNNKSDCKIQNSTGSFFWVNVKVNFDNVAMGYLALLQVATFKGWMDIMYAAVDSREVNMQPKWEDNVYMYLYFVIFIIFGGFFTLNLFVGVIIDNFNQQKKKLGGQDIFMTEEQKKYYNAMKKLGSKKPQKPIPRPLNKFQGFVFDIVTRQAFDITIMVLICLNMITMMVETDDQSEEKTKILGKINQFFVAVFTGECVMKMFALRQYYFTNGWNVFDFIVVVLSIASLIFSAILKSLQSYFSPTLFRVIRLARIGRILRLIRAAKGIRTLLFALMMSLPALFNIGLLLFLVMFIYSIFGMSSFPHVRWEAGIDDMFNFQTFANSMLCLFQITTSAGWDGLLSPILNTGPPYCDPNLPNSNGTRGDCGSPAVGIIFFTTYIIISFLIMVNMYIAVILENFNVATEESTEPLSEDDFDMFYETWEKFDPEATQFITFSALSDFADTLSGPLRIPKPNRNILIQMDLPLVPGDKIHCLDILFAFTKNVLGESGELDSLKANMEEKFMATNLSKSSYEPIATTLRWKQEDISATVIQKAYRSYVLHRSMALSNTPCVPRAEEEAASLPDEGFVAFTANENCVLPDKSETASATSFPPSYESVTRGLSDRVNMRTSSSIQNEDEATSMELIAPGP",
# GPC6_HUMAN
"MPSWIGAVILPLLGLLLSLPAGADVKARSCGEVRQAYGAKGFSLADIPYQEIAGEHLRICPQEYTCCTTEMEDKLSQQSKLEFENLVEETSHFVRTTFVSRHKKFDEFFRELLENAEKSLNDMFVRTYGMLYMQNSEVFQDLFTELKRYYTGGNVNLEEMLNDFWARLLERMFQLINPQYHFSEDYLECVSKYTDQLKPFGDVPRKLKIQVTRAFIAARTFVQGLTVGREVANRVSKVSPTPGCIRALMKMLYCPYCRGLPTVRPCNNYCLNVMKGCLANQADLDTEWNLFIDAMLLVAERLEGPFNIESVMDPIDVKISEAIMNMQENSMQVSAKVFQGCGQPKPAPALRSARSAPENFNTRFRPYNPEERPTTAAGTSLDRLVTDIKEKLKLSKKVWSALPYTICKDESVTAGTSNEEECWNGHSKARYLPEIMNDGLTNQINNPEVDVDITRPDTFIRQQIMALRVMTNKLKNAYNGNDVNFQDTSDESSGSGSGSGCMDDVCPTEFEFVTTEAPAVDPDRREVDSSAAQRGHSLLSWSLTCIVLALQRLCR",
# NPTN_HUMAN
"MSGSSLPSALALSLLLVSGSLLPGPGAAQNAGFVKSPMSETKLTGDAFELYCDVVGSPTPEIQWWYAEVNRAESFRQLWDGARKRRVTVNTAYGSNGVSVLRITRLTLEDSGTYECRASNDPKRNDLRQNPSITWIRAQATISVLQKPRIVTSEEVIIRDSPVLPVTLQCNLTSSSHTLTYSYWTKNGVELSATRKNASNMEYRINKPRAEDSGEYHCVYHFVSAPKANATIEVKAAPDITGHKRSENKNEGQDATMYCKSVGYPHPDWIWRKKENGMPMDIVNTSGRFFIINKENYTELNIVNLQITEDPGEYECNATNAIGSASVVTVLRVRSHLAPLWPFLGILAEIIILVVIIVVYEKRKRPDEVPDDDEPAGPMKTNSTNNHKDKNLRQRNTN",
# KPTN_HUMAN
"MMGEAAVAAGPCPLREDSFTRFSSQSNVYGLAGGAGGRGELLAATLKGKVLGFRYQDLRQKIRPVAKELQFNYIPVDAEIVSIDTFNKSPPKRGLVVGITFIKDSGDKGSPFLNIYCDYEPGSEYNLDSIAQSCLNLELQFTPFQLCHAEVQVGDQLETVFLLSGNDPAIHLYKENEGLHQFEEQPVENLFPELTNLTSSVLWLDVHNFPGTSRRLSALGCQSGYVRVAHVDQRSREVLQMWSVLQDGPISRVIVFSLSAAKETKDRPLQDEYSVLVASMLEPAVVYRDLLNRGLEDQLLLPGSDQFDSVLCSLVTDVDLDGRPEVLVATYGQELLCYKYRGPESGLPEAQHGFHLLWQRSFSSPLLAMAHVDLTGDGLQELAVVSLKGVHILQHSLIQASELVLTRLRHQVEQRRRRLQGLEDGAGAGPAENAAS",
# CCG2_HUMAN
"MGLFDRGVQMLLTTVGAFAAFSLMTIAVGTDYWLYSRGVCKTKSVSENETSKKNEEVMTHSGLWRTCCLEGNFKGLCKQIDHFPEDADYEADTAEYFLRAVRASSIFPILSVILLFMGGLCIAASEFYKTRHNIILSAGIFFVSAGLSNIIGIIVYISANAGDPSKSDSKKNSYSYGWSFYFGALSFIIAEMVGVLAVHMFIDRHKQLRATARATDYLQASAITRIPSYRYRYQRRSRSSSRSTEPSHSRDASPVGIKGFNTLPSTEISMYTLSRDPLKAATTPTATYNSDRDNSFLQVHNCIQKENKDSLHSNTANRRTTPV",
# TACC3_HUMAN
"MSLQVLNDKNVSNEKNTENCDFLFSPPEVTGRSSVLRVSQKENVPPKNLAKAMKVTFQTPLRDPQTHRILSPSMASKLEAPFTQDDTLGLENSHPVWTQKENQQLIKEVDAKTTHGILQKPVEADTDLLGDASPAFGSGSSSESGPGALADLDCSSSSQSPGSSENQMVSPGKVSGSPEQAVEENLSSYSLDRRVTPASETLEDPCRTESQHKAETPHGAEEECKAETPHGAEEECRHGGVCAPAAVATSPPGAIPKEACGGAPLQGLPGEALGCPAGVGTPVPADGTQTLTCAHTSAPESTAPTNHLVAGRAMTLSPQEEVAAGQMASSSRSGPVKLEFDVSDGATSKRAPPPRRLGERSGLKPPLRKAAVRQQKAPQEVEEDDGRSGAGEDPPMPASRGSYHLDWDKMDDPNFIPFGGDTKSGCSEAQPPESPETRLGQPAAEQLHAGPATEEPGPCLSQQLHSASAEDTPVVQLAAETPTAESKERALNSASTSLPTSCPGSEPVPTHQQGQPALELKEESFRDPAEVLGTGAEVDYLEQFGTSSFKESALRKQSLYLKFDPLLRDSPGRPVPVATETSSMHGANETPSGRPREAKLVEFDFLGALDIPVPGPPPGVPAPGGPPLSTGPIVDLLQYSQKDLDAVVKATQEENRELRSRCEELHGKNLELGKIMDRFEEVVYQAMEEVQKQKELSKAEIQKVLKEKDQLTTDLNSMEKSFSDLFKRFEKQKEVIEGYRKNEESLKKCVEDYLARITQEGQRYQALKAHAEEKLQLANEEIAQVRSKAQAEALALQASLRKEQMRIQSLEKTVEQKTKENEELTRICDDLISKMEKI",
# BIG2_HUMAN
"MQESQTKSMFVSRALEKILADKEVKRPQHSQLRRACQVALDEIKAEIEKQRLGTAAPPKANFIEADKYFLPFELACQSKSPRVVSTSLDCLQKLIAYGHITGNAPDSGAPGKRLIDRIVETICSCFQGPQTDEGVQLQIIKALLTAVTSPHIEIHEGTILQTVRTCYNIYLASKNLINQTTAKATLTQMLNVIFTRMENQVLQEARELEKPIQSKPQSPVIQAAAVSPKFVRLKHSQAQSKPTTPEKTDLTNGEHARSDSGKVSTENGDAPRERGSSLSGTDDGAQEVVKDILEDVVTSAIKEAAEKHGLTEPERVLGELECQECAIPPGVDENSQTNGIADDRQSLSSADNLESDAQGHQVAARFSHVLQKDAFLVFRSLCKLSMKPLGEGPPDPKSHELRSKVVSLQLLLSVLQNAGPVFRTHEMFINAIKQYLCVALSKNGVSSVPDVFELSLAIFLTLLSNFKMHLKMQIEVFFKEIFLNILETSTSSFEHRWMVIQTLTRICADAQCVVDIYVNYDCDLNAANIFERLVNDLSKIAQGRSGHELGMTPLQELSLRKKGLECLVSILKCMVEWSKDLYVNPNHQTSLGQERLTDQEIGDGKGLDMARRCSVTSMESTVSSGTQTTVQDDPEQFEVIKQQKEIIEHGIELFNKKPKRGIQFLQEQGMLGTSVEDIAQFLHQEERLDSTQVGDFLGDSARFNKEVMYAYVDQLDFCEKEFVSALRTFLEGFRLPGEAQKIDRLMEKFAARYIECNQGQTLFASADTAYVLAYSIIMLTTDLHSPQVKNKMTKEQYIKMNRGINDSKDLPEEYLSSIYEEIEGKKIAMKETKELTIATKSTKQNVASEKQRRLLYNLEMEQMAKTAKALMEAVSHAKAPFTSATHLDHVRPMFKLVWTPLLAAYSIGLQNCDDTEVASLCLEGIRCAIRIACIFGMQLERDAYVQALARFSLLTASSSITEMKQKNIDTIKTLITVAHTDGNYLGNSWHEILKCISQLELAQLIGTGVKTRYLSGSGREREGSLKGHTLAGEEFMGLGLGNLVSGGVDKRQMASFQESVGETSSQSVVVAVDRIFTGSTRLDGNAIVDFVRWLCAVSMDELASPHHPRMFSLQKIVEISYYNMNRIRLQWSRIWHVIGDHFNKVGCNPNEDVAIFAVDSLRQLSMKFLEKGELANFRFQKDFLRPFEHIMKKNRSPTIRDMAIRCIAQMVNSQAANIRSGWKNIFAVFHQAASDHDGNIVELAFQTTCHIVTTIFQHHFPAAIDSFQDAVKCLSEFACNAAFPDTSMEAIRLIRFCGKYVSERPRVLQEYTSDDMNVAPGDRVWVRGWFPILFELSCIINRCKLDVRTRGLTVMFEIMKSYGHTFEKHWWQDLFRIVFRIFDNMKLPEQLSEKSEWMTTTCNHALYAICDVFTQFYEALNEVLLSDVFAQLQWCVKQDNEQLARSGTNCLENLVISNGEKFSPEVWDETCNCMLDIFKTTIPHVLLTWRPVGMEEDSSEKHLDVDLDRQSLSSIDKNPSERGQSQLSNPTDDSWKGRPYANQKLFASLLIKCVVQLELIQTIDNIVFYPATSKKEDAEHMVAAQQDTLDADIHIETEDQGMYKYMSSQHLFKLLDCLQESHSFSKAFNSNYEQRTVLWRAGFKGKSKPNLLKQETSSLACCLRILFRMYVDENRRDSWEEIQQRLLTVCSEALAYFITVNSESHREAWTSLLLLLLTKTLKINDEKFKAHASMYYPYLCEIMQFDLIPELRAVLRKFFLRIGVVYKIWIPEEPSQVPAALSPVW",
# EPN1_HUMAN
"MSTSSLRRQMKNIVHNYSEAEIKVREATSNDPWGPSSSLMSEIADLTYNVVAFSEIMSMIWKRLNDHGKNWRHVYKAMTLMEYLIKTGSERVSQQCKENMYAVQTLKDFQYVDRDGKDQGVNVREKAKQLVALLRDEDRLREERAHALKTKEKLAQTATASSAAVGSGPPPEAEQAWPQSSGEEELQLQLALAMSKEEADQPPSCGPEDDAQLQLALSLSREEHDKEERIRRGDDLRLQMAIEESKRETGGKEESSLMDLADVFTAPAPAPTTDPWGGPAPMAAAVPTAAPTSDPWGGPPVPPAADPWGGPAPTPASGDPWRPAAPAGPSVDPWGGTPAPAAGEGPTPDPWGSSDGGVPVSGPSASDPWTPAPAFSDPWGGSPAKPSTNGTTAAGGFDTEPDEFSDFDRLRTALPTSGSSAGELELLAGEVPARSPGAFDMSGVRGSLAEAVGSPPPAATPTPTPPTRKTPESFLGPNAALVDLDSLVSRPGPTPPGAKASNPFLPGGGPATGPSVTNPFQPAPPATLTLNQLRLSPVPPVPGAPPTYISPLGGGPGLPPMMPPGPPAPNTNPFLL",
# ZNT1_HUMAN
"MGCWGRNRGRLLCMLALTFMFMVLEVVVSRVTSSLAMLSDSFHMLSDVLALVVALVAERFARRTHATQKNTFGWIRAEVMGALVNAIFLTGLCFAILLEAIERFIEPHEMQQPLVVLGVGVAGLLVNVLGLCLFHHHSGFSQDSGHGHSHGGHGHGHGLPKGPRVKSTRPGSSDINVAPGEQGPDQEETNTLVANTSNSNGLKLDPADPENPRSGDTVEVQVNGNLVREPDHMELEEDRAGQLNMRGVFLHVLGDALGSVIVVVNALVFYFSWKGCSEGDFCVNPCFPDPCKAFVEIINSTHASVYEAGPCWVLYLDPTLCVVMVCILLYTTYPLLKESALILLQTVPKQIDIRNLIKELRNVEGVEEVHELHVWQLAGSRIIATAHIKCEDPTSYMEVAKTIKDVFHNHGIHATTIQPEFASVGSKSSVVPCELACRTQCALKQCCGTLPQAPSGKDAEKTPAVSISCLELSNNLEKKPRRTKAENIPAVVIEIKNMPNKQPESSL",
# CAD10_HUMAN
"MTIHQFLLLFLFWVCLPHFCSPEIMFRRTPVPQQRILSSRVPRSDGKILHRQKRGWMWNQFFLLEEYTGSDYQYVGKLHSDQDKGDGSLKYILSGDGAGTLFIIDEKTGDIHATRRIDREEKAFYTLRAQAINRRTLRPVEPESEFVIKIHDINDNEPTFPEEIYTASVPEMSVVGTSVVQVTATDADDPSYGNSARVIYSILQGQPYFSVEPETGIIRTALPNMNRENREQYQVVIQAKDMGGQMGGLSGTTTVNITLTDVNDNPPRFPQNTIHLRVLESSPVGTAIGSVKATDADTGKNAEVEYRIIDGDGTDMFDIVTEKDTQEGIITVKKPLDYESRRLYTLKVEAENTHVDPRFYYLGPFKDTTIVKISIEDVDEPPVFSRSSYLFEVHEDIEVGTIIGTVMARDPDSISSPIRFSLDRHTDLDRIFNIHSGNGSLYTSKPLDRELSQWHNLTVIAAEINNPKETTRVAVFVRILDVNDNAPQFAVFYDTFVCENARPGQLIQTISAVDKDDPLGGQKFFFSLAAVNPNFTVQDNEDNTARILTRKNGFNRHEISTYLLPVVISDNDYPIQSSTGTLTIRVCACDSQGNMQSCSAEALLLPAGLSTGALIAILLCIIILLVIVVLFAALKRQRKKEPLILSKEDIRDNIVSYNDEGGGEEDTQAFDIGTLRNPAAIEEKKLRRDIIPETLFIPRRTPTAPDNTDVRDFINERLKEHDLDPTAPPYDSLATYAYEGNDSIAESLSSLESGTTEGDQNYDYLREWGPRFNKLAEMYGGGESDKDS",
# NCOA3_HUMAN
"MSGLGENLDPLASDSRKRKLPCDTPGQGLTCSGEKRRREQESKYIEELAELISANLSDIDNFNVKPDKCAILKETVRQIRQIKEQGKTISNDDDVQKADVSSTGQGVIDKDSLGPLLLQALDGFLFVVNRDGNIVFVSENVTQYLQYKQEDLVNTSVYNILHEEDRKDFLKNLPKSTVNGVSWTNETQRQKSHTFNCRMLMKTPHDILEDINASPEMRQRYETMQCFALSQPRAMMEEGEDLQSCMICVARRITTGERTFPSNPESFITRHDLSGKVVNIDTNSLRSSMRPGFEDIIRRCIQRFFSLNDGQSWSQKRHYQEAYLNGHAETPVYRFSLADGTIVTAQTKSKLFRNPVTNDRHGFVSTHFLQREQNGYRPNPNPVGQGIRPPMAGCNSSVGGMSMSPNQGLQMPSSRAYGLADPSTTGQMSGARYGGSSNIASLTPGPGMQSPSSYQNNNYGLNMSSPPHGSPGLAPNQQNIMISPRNRGSPKIASHQFSPVAGVHSPMASSGNTGNHSFSSSSLSALQAISEGVGTSLLSTLSSPGPKLDNSPNMNITQPSKVSNQDSKSPLGFYCDQNPVESSMCQSNSRDHLSDKESKESSVEGAENQRGPLESKGHKKLLQLLTCSSDDRGHSSLTNSPLDSSCKESSVSVTSPSGVSSSTSGGVSSTSNMHGSLLQEKHRILHKLLQNGNSPAEVAKITAEATGKDTSSITSCGDGNVVKQEQLSPKKKENNALLRYLLDRDDPSDALSKELQPQVEGVDNKMSQCTSSTIPSSSQEKDPKIKTETSEEGSGDLDNLDAILGDLTSSDFYNNSISSNGSHLGTKQQVFQGTNSLGLKSSQSVQSIRPPYNRAVSLDSPVSVGSSPPVKNISAFPMLPKQPMLGGNPRMMDSQENYGSSMGGPNRNVTVTQTPSSGDWGLPNSKAGRMEPMNSNSMGRPGGDYNTSLPRPALGGSIPTLPLRSNSIPGARPVLQQQQQMLQMRPGEIPMGMGANPYGQAAASNQLGSWPDGMLSMEQVSHGTQNRPLLRNSLDDLVGPPSNLEGQSDERALLDQLHTLLSNTDATGLEEIDRALGIPELVNQGQALEPKQDAFQGQEAAVMMDQKAGLYGQTYPAQGPPMQGGFHLQGQSPSFNSMMNQMNQQGNFPLQGMHPRANIMRPRTNTPKQLRMQLQQRLQGQQFLNQSRQALELKMENPTAGGAAVMRPMMQPQVSSQQGFLNAQMVAQRSRELLSHHFRQQRVAMMMQQQQQQQQQQQQQQQQQQQQQQQQQQQQQTQAFSPPPNVTASPSMDGLLAGPTMPQAPPQQFPYQPNYGMGQQPDPAFGRVSSPPNAMMSSRMGPSQNPMMQHPQAASIYQSSEMKGWPSGNLARNSSFSQQQFAHQGNPAVYSMVHMNGSSGHMGQMNMNPMPMSGMPMGPDQKYC",
# NUMBL_HUMAN
"MSRSAAASGGPRRPERHLPPAPCGAPGPPETCRTEPDGAGTMNKLRQSLRRRKPAYVPEASRPHQWQADEDAVRKGTCSFPVRYLGHVEVEESRGMHVCEDAVKKLKAMGRKSVKSVLWVSADGLRVVDDKTKDLLVDQTIEKVSFCAPDRNLDKAFSYICRDGTTRRWICHCFLALKDSGERLSHAVGCAFAACLERKQRREKECGVTAAFDASRTSFAREGSFRLSGGGRPAEREAPDKKKAEAAAAPTVAPGPAQPGHVSPTPATTSPGEKGEAGTPVAAGTTAAAIPRRHAPLEQLVRQGSFRGFPALSQKNSPFKRQLSLRLNELPSTLQRRTDFQVKGTVPEMEPPGAGDSDSINALCTQISSSFASAGAPAPGPPPATTGTSAWGEPSVPPAAAFQPGHKRTPSEAERWLEEVSQVAKAQQQQQQQQQQQQQQQQQQQQAASVAPVPTMPPALQPFPAPVGPFDAAPAQVAVFLPPPHMQPPFVPAYPGLGYPPMPRVPVVGITPSQMVANAFCSAAQLQPQPATLLGKAGAFPPPAIPSAPGSQARPRPNGAPWPPEPAPAPAPELDPFEAQWAALEGKATVEKPSNPFSGDLQKTFEIEL",
# DGKB_HUMAN
"MTNQEKWAHLSPSEFSQLQKYAEYSTKKLKDVLEEFHGNGVLAKYNPEGKQDILNQTIDFEGFKLFMKTFLEAELPDDFTAHLFMSFSNKFPHSSPMVKSKPALLSGGLRMNKGAITPPRTTSPANTCSPEVIHLKDIVCYLSLLERGRPEDKLEFMFRLYDTDGNGFLDSSELENIISQMMHVAEYLEWDVTELNPILHEMMEEIDYDHDGTVSLEEWIQGGMTTIPLLVLLGLENNVKDDGQHVWRLKHFNKPAYCNLCLNMLIGVGKQGLCCSFCKYTVHERCVARAPPSCIKTYVKSKRNTDVMHHYWVEGNCPTKCDKCHKTVKCYQGLTGLHCVWCQITLHNKCASHLKPECDCGPLKDHILPPTTICPVVLQTLPTSGVSVPEERQSTVKKEKSGSQQPNKVIDKNKMQRANSVTVDGQGLQVTPVPGTHPLLVFVNPKSGGKQGERIYRKFQYLLNPRQVYSLSGNGPMPGLNFFRDVPDFRVLACGGDGTVGWVLDCIEKANVGKHPPVAILPLGTGNDLARCLRWGGGYEGENLMKILKDIENSTEIMLDRWKFEVIPNDKDEKGDPVPYSIINNYFSIGVDASIAHRFHIMREKHPEKFNSRMKNKFWYFEFGTSETFSATCKKLHESVEIECDGVQIDLINISLEGIAILNIPSMHGGSNLWGESKKRRSHRRIEKKGSDKRTTVTDAKELKFASQDLSDQLLEVVGLEGAMEMGQIYTGLKSAGRRLAQCSCVVIRTSKSLPMQIDGEPWMQTPCTIKITHKNQAPMLMGPPPKTGLFCSLVKRTRNRSKE",
# PCLO_HUMAN
"MGNEASLEGEGLPEGLAAAAAAGGGASGAGSPSHTAIPAGMEADLSQLSEEERRQIAAVMSRAQGLPKGSVPPAAAESPSMHRKQELDSSHPPKQSGRPPDPGRPAQPGLSKSRTTDTFRSEQKLPGRSPSTISLKESKSRTDLKEEHKSSMMPGFLSEVNALSAVSSVVNKFNPFDLISDSEASQEETTKKQKVVQKEQGKPEGIIKPPLQQQPPKPIPKQQGPGRDPLQQDGTPKSISSQQPEKIKSQPPGTGKPIQGPTQTPQTDHAKLPLQRDASRPQTKQADIVRGESVKPSLPSPSKPPIQQPTPGKPPAQQPGHEKSQPGPAKPPAQPSGLTKPLAQQPGTVKPPVQPPGTTKPPAQPLGPAKPPAQQTGSEKPSSEQPGPKALAQPPGVGKTPAQQPGPAKPPTQQVGTPKPLAQQPGLQSPAKAPGPTKTPVQQPGPGKIPAQQAGPGKTSAQQTGPTKPPSQLPGPAKPPPQQPGPAKPPPQQPGSAKPPSQQPGSTKPPPQQPGPAKPSPQQPGSTKPPSQQPGSAKPSAQQPSPAKPSAQQSTKPVSQTGSGKPLQPPTVSPSAKQPPSQGLPKTICPLCNTTELLLHVPEKANFNTCTECQTTVCSLCGFNPNPHLTEVKEWLCLNCQMKRALGGDLAPVPSSPQPKLKTAPVTTTSAVSKSSPQPQQTSPKKDAAPKQDLSKAPEPKKPPPLVKQPTLHGSPSAKAKQPPEADSLSKPAPPKEPSVPSEQDKAPVADDKPKQPKMVKPTTDLVSSSSATTKPDIPSSKVQSQAEEKTTPPLKTDSAKPSQSFPPTGEKVSPFDSKAIPRPASDSKIISHPGPSSESKGQKQVDPVQKKEEPKKAQTKMSPKPDAKPMPKGSPTPPGPRPTAGQTVPTPQQSPKPQEQSRRFSLNLGSITDAPKSQPTTPQETVTGKLFGFGASIFSQASNLISTAGQPGPHSQSGPGAPMKQAPAPSQPPTSQGPPKSTGQAPPAPAKSIPVKKETKAPAAEKLEPKAEQAPTVKRTETEKKPPPIKDSKSLTAEPQKAVLPTKLEKSPKPESTCPLCKTELNIGSKDPPNFNTCTECKNQVCNLCGFNPTPHLTEIQEWLCLNCQTQRAISGQLGDIRKMPPAPSGPKASPMPVPTESSSQKTAVPPQVKLVKKQEQEVKTEAEKVILEKVKETLSMEKIPPMVTTDQKQEESKLEKDKASALQEKKPLPEEKKLIPEEEKIRSEEKKPLLEEKKPTPEDKKLLPEAKTSAPEEQKHDLLKSQVQIAEEKLEGRVAPKTVQEGKQPQTKMEGLPSGTPQSLPKEDDKTTKTIKEQPQPPCTAKPDQVEPGKEKTEKEDDKSDTSSSQQPKSPQGLSDTGYSSDGISSSLGEIPSLIPTDEKDILKGLKKDSFSQESSPSSPSDLAKLESTVLSILEAQASTLADEKSEKKTQPHEVSPEQPKDQEKTQSLSETLEITISEEEIKESQEERKDTFKKDSQQDIPSSKDHKEKSEFVDDITTRREPYDSVEESSESENSPVPQRKRRTSVGSSSSDEYKQEDSQGSGEEEDFIRKQIIEMSADEDASGSEDDEFIRNQLKEISSSTESQKKEETKGKGKITAGKHRRLTRKSSTSIDEDAGRRHSWHDEDDEAFDESPELKYRETKSQESEELVVTGGGGLRRFKTIELNSTIADKYSAESSQKKTSLYFDEEPELEMESLTDSPEDRSRGEGSSSLHASSFTPGTSPTSVSSLDEDSDSSPSHKKGESKQQRKARHRPHGPLLPTIEDSSEEEELREEEELLKEQEKQREIEQQQRKSSSKKSKKDKDELRAQRRRERPKTPPSNLSPIEDASPTEELRQAAEMEELHRSSCSEYSPSIESDPEGFEISPEKIIEVQKVYKLPTAVSLYSPTDEQSIMQKEGSQKALKSAEEMYEEMMHKTHKYKAFPAANERDEVFEKEPLYGGMLIEDYIYESLVEDTYNGSVDGSLLTRQEEENGFMQQKGREQKIRLSEQIYEDPMQKITDLQKEFYELESLHSVVPQEDIVSSSFIIPESHEIVDLGTMVTSTEEERKLLDADAAYEELMKRQQMQLTPGSSPTQAPIGEDMTESTMDFDRMPDASLTSSVLSGASLTDSTSSATLSIPDVKITQHFSTEEIEDEYVTDYTREIQEIIAHESLILTYSEPSESATSVPPSDTPSLTSSVSSVCTTDSSSPITTLDSITTVYTEPVDMITKFEDSEEISSSTYFPGSIIDYPEEISVSLDRTAPPDGRASADHIVISLSDMASSIIESVVPKPEGPVADTVSTDLLISEKDPVKKAKKETGNGIILEVLEAYRDKKELEAERTKSSLSETVFDHPPSSVIALPMKEQLSTTYFTSGETFGQEKPASQLPSGSPSVSSLPAKPRPFFRSSSLDISAQPPPPPPPPPPPPPPPPPPPPPPLPPPTSPKPTILPKKKLTVASPVTTATPLFDAVTTLETTAVLRSNGLPVTRICTTAPPPVPPKPSSIPSGLVFTHRPEPSKPPIAPKPVIPQLPTTTQKPTDIHPKPTGLSLTSSMTLNLVTSADYKLPSPTSPLSPHSNKSSPRFSKSLTETYVVITLPSEPGTPTDSSASQAITSWPLGSPSKDLVSVEPVFSVVPPVTAVEIPISSEQTFYISGALQTFSATPVTAPSSFQAAPTSVTQFLTTEVSKTEVSATRSTAPSVGLSSISITIPPEPLALDNIHLEKPQYKEDGKLQLVGDVIDLRTVPKVEVKTTDKCIDLSASTMDVKRQITANEVYGKQISAVQPSIINLSVTSSIVTPVSLATETVTFVTCTASASYTTGTESLVGAEHAMTTPLQLTTSKHAEPPYRIPSDQVFPIAREEAPINLSLGTPAHAVTLAITKPVTVPPVGVTNGWTDSTVSQGITDGEVVDLSTTKSHRTVVTMDESTSSVMTKIIEDEKPVDLTAGRRAVCCDVVYKLPFGRSCTAQQPATTLPEDRFGYRDDHYQYDRSGPYGYRGIGGMKPSMSDTNLAEAGHFFYKSKNAFDYSEGTDTAVDLTSGRVTTGEVMDYSSKTTGPYPETRQVISGAGISTPQYSTARMTPPPGPQYCVGSVLRSSNGVVYSSVATPTPSTFAITTQPGSIFSTTVRDLSGIHTADAVTSLPAMHHSQPMPRSYFITTGASETDIAVTGIDISASLQTITMESLTAETIDSVPTLTTASEVFPEVVGDESALLIVPEEDKQQQQLDLERELLELEKIKQQRFAEELEWERQEIQRFREQEKIMVQKKLEELQSMKQHLLFQQEEERQAQFMMRQETLAQQQLQLEQIQQLQQQLHQQLEEQKIRQIYQYNYDPSGTASPQTTTEQAILEGQYAALEGSQFWATEDATTTASAVVAIEIPQSQGWYTVQSDGVTQYIAPPGILSTVSEIPLTDVVVKEEKQPKKRSSGAKVRGQYDDMGENMTDDPRSFKKIVDSGVQTDDEDATDRSYVSRRRRTKKSVDTSVQTDDEDQDEWDMPTRSRRKARVGKYGDSMTEADKTKPLSKVSSIAVQTVAEISVQTEPVGTIRTPSIRARVDAKVEIIKHISAPEKTYKGGSLGCQTEADSDTQSPQYLSATSPPKDKKRPTPLEIGYSSHLRADSTVQLAPSPPKSPKVLYSPISPLSPGKALESAFVPYEKPLPDDISPQKVLHPDMAKVPPASPKTAKMMQRSMSDPKPLSPTADESSRAPFQYTEGYTTKGSQTMTSSGAQKKVKRTLPNPPPEEISTGTQSTFSTMGTVSRRRICRTNTMARAKILQDIDRELDLVERESAKLRKKQAELDEEEKEIDAKLRYLEMGINRRKEALLKEREKRERAYLQGVAEDRDYMSDSEVSSTRPTRIESQHGIERPRTAPQTEFSQFIPPQTQTESQLVPPTSPYTQYQYSSPALPTQAPTSYTQQSHFEQQTLYHQQVSPYQTQPTFQAVATMSFTPQVQPTPTPQPSYQLPSQMMVIQQKPRQTTLYLEPKITSNYEVIRNQPLMIAPVSTDNTFAVSHLGSKYNSLDLRIGLEERSSMASSPISSISADSFYADIDHHTPRNYVLIDDIGEITKGTAALSTAFSLHEKDLSKTDRLLRTTETRRSQEVTDFLAPLQSSSRLHSYVKAEEDPMEDPYELKLLKHQIKQEFRRGTESLDHLAGLSHYYHADTSYRHFPKSEKYSISRLTLEKQAAKQLPAAILYQKQSKHKKSLIDPKMSKFSPIQESRDLEPDYSSYMTSSTSSIGGISSRARLLQDDITFGLRKNITDQQKFMGSSLGTGLGTLGNTIRSALQDEADKPYSSGSRSRPSSRPSSVYGLDLSIKRDSSSSSLRLKAQEAEALDVSFSHASSSARTKPTSLPISQSRGRIPIVAQNSEEESPLSPVGQPMGMARAAAGPLPPISADTRDQFGSSHSLPEVQQHMREESRTRGYDRDIAFIMDDFQHAMSDSEAYHLRREETDWFDKPRESRLENGHGLDRKLPERLVHSRPLSQHQEQIIQMNGKTMHYIFPHARIKITRDSKDHTVSGNGLGIRIVGGKEIPGHSGEIGAYIAKILPGGSAEQTGKLMEGMQVLEWNGIPLTSKTYEEVQSIISQQSGEAEICVRLDLNMLSDSENSQHLELHEPPKAVDKAKSPGVDPKQLAAELQKVSLQQSPLVLSSVVEKGSHVHSGPTSAGSSSVPSPGQPGSPSVSKKKHGSSKPTDGTKVVSHPITGEIQLQINYDLGNLIIHILQARNLVPRDNNGYSDPFVKVYLLPGRGQVMVVQNASAEYKRRTKHVQKSLNPEWNQTVIYKSISMEQLKKKTLEVTVWDYDRFSSNDFLGEVLIDLSSTSHLDNTPRWYPLKEQTESIDHGKSHSSQSSQQSPKPSVIKSRSHGIFPDPSKDMQVPTIEKSHSSPGSSKSSSEGHLRSHGPSRSQSKTSVTQTHLEDAGAAIAAAEAAVQQLRIQPTKPPNHRPAESSVSTGSSGSSFGSGYSVDSEGSSSTAGETNLFPIPRIGKMGQNGQEPVKQPGVGVGLADTEAKTQVMGEIKIALKKEMKTDGEQLIVEILQCRNITYKFKSPDHLPDLYVKIYVMNISTQKKVIKKKTRVCRHDREPSFNETFRFSLSPAGHSLQILLFSNGGKFMKKTLIGEACIWLDKVDLRKRIVNWHKLLVSPTQTH",
# WASF2_HUMAN
"MPLVTRNIEPRHLCRQTLPSVRSELECVTNITLANVIRQLGSLSKYAEDIFGELFTQANTFASRVSSLAERVDRLQVKVTQLDPKEEEVSLQGINTRKAFRSSTIQDQKLFDRNSLPVPVLETYNTCDTPPPLNNLTPYRDDGKEALKFYTDPSYFFDLWKEKMLQDTKDIMKEKRKHRKEKKDNPNRGNVNPRKIKTRKEEWEKMKMGQEFVESKEKLGTSGYPPTLVYQNGSIGCVENVDASSYPPPPQSDSASSPSPSFSEDNLPPPPAEFSYPVDNQRGSGLAGPKRSSVVSPSHPPPAPPLGSPPGPKPGFAPPPAPPPPPPPMIGIPPPPPPVGFGSPGTPPPPSPPSFPPHPDFAAPPPPPPPPAADYPTLPPPPLSQPTGGAPPPPPPPPPPGPPPPPFTGADGQPAIPPPLSDTTKPKSSLPAVSDARSDLLSAIRQGFQLRRVEEQREQEKRDVVGNDVATILSRRIAVEYSDSEDDSSEFDEDDWSD",
# PIAS3_HUMAN
"MAELGELKHMVMSFRVSELQVLLGFAGRNKSGRKHELLAKALHLLKSSCAPSVQMKIKELYRRRFPRKTLGPSDLSLLSLPPGTSPVGSPGPLAPIPPTLLAPGTLLGPKREVDMHPPLPQPVHPDVTMKPLPFYEVYGELIRPTTLASTSSQRFEEAHFTFALTPQQVQQILTSREVLPGAKCDYTIQVQLRFCLCETSCPQEDYFPPNLFVKVNGKLCPLPGYLPPTKNGAEPKRPSRPINITPLARLSATVPNTIVVNWSSEFGRNYSLSVYLVRQLTAGTLLQKLRAKGIRNPDHSRALIKEKLTADPDSEVATTSLRVSLMCPLGKMRLTVPCRALTCAHLQSFDAALYLQMNEKKPTWTCPVCDKKAPYESLIIDGLFMEILSSCSDCDEIQFMEDGSWCPMKPKKEASEVCPPPGYGLDGLQYSPVQGGDPSENKKKVEVIDLTIESSSDEEDLPPTKKHCSVTSAAIPALPGSKGVLTSGHQPSSVLRSPAMGTLGGDFLSSLPLHEYPPAFPLGADIQGLDLFSFLQTESQHYGPSVITSLDEQDALGHFFQYRGTPSHFLGPLAPTLGSSHCSATPAPPPGRVSSIVAPGGALREGHGGPLPSGPSLTGCRSDIISLD"
]


# Add protein names corresponding to the sequences
protein_names = [
"RPSA2_HUMAN",
"GRD2I_HUMAN",
"PSD1_HUMAN",
"WIPF3_HUMAN",
"SHSA7_HUMAN",
"RHG32_HUMAN",
"GBRR3_HUMAN",
"MYO9A_HUMAN",
"SHSA9_HUMAN",
"OSCRI_HUMAN",
"APRIO_HUMAN",
"KIF2A_HUMAN",
"FZD9_HUMAN",
"SNP23_HUMAN",
"KCNK1_HUMAN",
"RB27B_HUMAN",
"APBB1_HUMAN",
"GRM8_HUMAN",
"MEN1_HUMAN",
"HIP1_HUMAN",
"EIF3F_HUMAN",
"CACB4_HUMAN",
"EAA5_HUMAN",
"WASL_HUMAN",
"EF2K_HUMAN",
"DNM1L_HUMAN",
"SYT5_HUMAN",
"AGRIN_HUMAN",
"BIN1_HUMAN",
"FAAH1_HUMAN",
"CAC1A_HUMAN",
"SDCB1_HUMAN",
"DDX3X_HUMAN",
"PODXL_HUMAN",
"WNT7A_HUMAN",
"DLGP1_HUMAN",
"NRG2_HUMAN",
"AGRB1_HUMAN",
"PTPRT_HUMAN",
"FCHO1_HUMAN",
"RHG33_HUMAN",
"NCAN_HUMAN",
"IF1AY_HUMAN",
"AP3D1_HUMAN",
"ABLM1_HUMAN",
"DVL1_HUMAN",
"STX16_HUMAN",
"ADA10_HUMAN",
"GBRD_HUMAN",
"GNB5_HUMAN",
"NRP1_HUMAN",
"UN13B_HUMAN",
"CPLX1_HUMAN",
"RGRF2_HUMAN",
"GIPC1_HUMAN",
"LIN7A_HUMAN",
"PCD17_HUMAN",
"RGS12_HUMAN",
"CSKP_HUMAN",
"AURKA_HUMAN",
"HNRDL_HUMAN",
"SYN3_HUMAN",
"SPTN2_HUMAN",
"RIMB2_HUMAN",
"SYNJ2_HUMAN",
"KIF3B_HUMAN",
"DCLK1_HUMAN",
"SNPH_HUMAN",
"ERC2_HUMAN",
"SCAM1_HUMAN",
"NPFF_HUMAN",
"ARPC2_HUMAN",
"ARPC3_HUMAN",
"AXIN1_HUMAN",
"PGRC2_HUMAN",
"LAMA5_HUMAN",
"VGF_HUMAN",
"S22A2_HUMAN",
"OGT1_HUMAN",
"EIF3D_HUMAN",
"NCAM2_HUMAN",
"NMDE4_HUMAN",
"STX7_HUMAN",
"ARPC5_HUMAN",
"P2RX6_HUMAN",
"FLRT2_HUMAN",
"PLXB1_HUMAN",
"PJA2_HUMAN",
"SI1L1_HUMAN",
"SEPT4_HUMAN",
"SRGP3_HUMAN",
"LRRT2_HUMAN",
"ARHG9_HUMAN",
"GRID2_HUMAN",
"SYNJ1_HUMAN",
"IF4G3_HUMAN",
"TGON2_HUMAN",
"KCNQ3_HUMAN",
"KCNQ2_HUMAN",
"RGS14_HUMAN",
"SYT7_HUMAN",
"DCX_HUMAN",
"OREX_HUMAN",
"AKA7A_HUMAN",
"ACTN4_HUMAN",
"AP1G1_HUMAN",
"STX6_HUMAN",
"SNG1_HUMAN",
"SNG3_HUMAN",
"SGTA_HUMAN",
"STRN_HUMAN",
"KALRN_HUMAN",
"AGRB2_HUMAN",
"AGRB3_HUMAN",
"PCDH7_HUMAN",
"PRKN_HUMAN",
"GBG7_HUMAN",
"KIF5C_HUMAN",
"SI1L3_HUMAN",
"LZTS3_HUMAN",
"PI51C_HUMAN",
"KIF1B_HUMAN",
"CCG3_HUMAN",
"NMD3B_HUMAN",
"EVI5_HUMAN",
"NRP2_HUMAN",
"BRNP1_HUMAN",
"PLXC1_HUMAN",
"HNRPQ_HUMAN",
"RNBP6_HUMAN",
"TDRD6_HUMAN",
"DIAP1_HUMAN",
"AP180_HUMAN",
"EXOC3_HUMAN",
"MOT2_HUMAN",
"JAK2_HUMAN",
"SRPX2_HUMAN",
"CTND1_HUMAN",
"HCN1_HUMAN",
"PRAF2_HUMAN",
"IF2P_HUMAN",
"OPHN1_HUMAN",
"RAMP1_HUMAN",
"NOL3_HUMAN",
"DTNB_HUMAN",
"KI21B_HUMAN",
"SRGP2_HUMAN",
"CAPON_HUMAN",
"AUXI_HUMAN",
"ADA23_HUMAN",
"WDR1_HUMAN",
"LRP4_HUMAN",
"ROCK2_HUMAN",
"CLAP2_HUMAN",
"LIPA3_HUMAN",
"HIP1R_HUMAN",
"RFIP3_HUMAN",
"GLRA3_HUMAN",
"SEM7A_HUMAN",
"LIPA2_HUMAN",
"LIPA4_HUMAN",
"VATG1_HUMAN",
"BCAS1_HUMAN",
"VAMP4_HUMAN",
"TRIM3_HUMAN",
"SC22B_HUMAN",
"GPC4_HUMAN",
"DAB1_HUMAN",
"KS6A5_HUMAN",
"KS6A4_HUMAN",
"PALM_HUMAN",
"RENR_HUMAN",
"AP1G2_HUMAN",
"GABR2_HUMAN",
"DGKI_HUMAN",
"PAK3_HUMAN",
"PRAF3_HUMAN",
"RGS9_HUMAN",
"PIAS1_HUMAN",
"FLOT1_HUMAN",
"TRIO_HUMAN",
"MPDZ_HUMAN",
"C1QRF_HUMAN",
"WFS1_HUMAN",
"CDKL5_HUMAN",
"NEUL1_HUMAN",
"CNTN5_HUMAN",
"BAIP3_HUMAN",
"SLIT2_HUMAN",
"PKHG5_HUMAN",
"DEND_HUMAN",
"AGRL1_HUMAN",
"ABLM3_HUMAN",
"SLIK3_HUMAN",
"AP2A2_HUMAN",
"CSTN1_HUMAN",
"ARHGF_HUMAN",
"SLIK5_HUMAN",
"KCNK2_HUMAN",
"S1PR2_HUMAN",
"RIMB1_HUMAN",
"NXPH4_HUMAN",
"GBRAP_HUMAN",
"CAC1H_HUMAN",
"CSPG5_HUMAN",
"RTN3_HUMAN",
"PCDH8_HUMAN",
"EPN2_HUMAN",
"SNX4_HUMAN",
"KCNH1_HUMAN",
"VAPB_HUMAN",
"SNAPN_HUMAN",
"ATG7_HUMAN",
"PARN_HUMAN",
"GNAS3_HUMAN",
"AGRL2_HUMAN",
"NPTXR_HUMAN",
"NET1_HUMAN",
"VATG2_HUMAN",
"SNP29_HUMAN",
"CPNE6_HUMAN",
"SEM4F_HUMAN",
"AP2A1_HUMAN",
"STAU1_HUMAN",
"LATS1_HUMAN",
"DLGP3_HUMAN",
"NOE2_HUMAN",
"LGI1_HUMAN",
"S1PR4_HUMAN",
"NUDT3_HUMAN",
"F107A_HUMAN",
"APCL_HUMAN",
"CLPT1_HUMAN",
"NAPSA_HUMAN",
"ABL1_HUMAN",
"EGFR_HUMAN",
"PLMN_HUMAN",
"TPA_HUMAN",
"RASH_HUMAN",
"ANF_HUMAN",
"NEU1_HUMAN",
"NEU2_HUMAN",
"PENK_HUMAN",
"PDYN_HUMAN",
"CALC_HUMAN",
"NPY_HUMAN",
"APOE_HUMAN",
"APOC2_HUMAN",
"FIBA_HUMAN",
"FIBB_HUMAN",
"FIBG_HUMAN",
"ACHA_HUMAN",
"C1QA_HUMAN",
"C1QB_HUMAN",
"C1QC_HUMAN",
"TFR1_HUMAN",
"ANXA1_HUMAN",
"GCR_HUMAN",
"PRIO_HUMAN",
"THY1_HUMAN",
"G3P_HUMAN",
"ERBB2_HUMAN",
"GNAI2_HUMAN",
"IGF1_HUMAN",
"A4_HUMAN",
"ITB3_HUMAN",
"KPCG_HUMAN",
"IF2A_HUMAN",
"RLA1_HUMAN",
"RLA2_HUMAN",
"RLA0_HUMAN",
"ITB1_HUMAN",
"KPCB_HUMAN",
"CALB1_HUMAN",
"LCK_HUMAN",
"FYN_HUMAN",
"CCKN_HUMAN",
"APOA4_HUMAN",
"IF4E_HUMAN",
"CRF_HUMAN",
"CALCA_HUMAN",
"NFL_HUMAN",
"NFM_HUMAN",
"CATD_HUMAN",
"CAN1_HUMAN",
"TBB5_HUMAN",
"GRP_HUMAN",
"CASK_HUMAN",
"ACHG_HUMAN",
"ADRB2_HUMAN",
"PROF1_HUMAN",
"HS90A_HUMAN",
"YES_HUMAN",
"LYN_HUMAN",
"IGF1R_HUMAN",
"TNR16_HUMAN",
"ACM2_HUMAN",
"ACM4_HUMAN",
"4F2_HUMAN",
"MCR_HUMAN",
"SYPH_HUMAN",
"ADRB1_HUMAN",
"ITA5_HUMAN",
"RS17_HUMAN",
"ANXA5_HUMAN",
"RSSA_HUMAN",
"5HT1A_HUMAN",
"ADA2A_HUMAN",
"GNAO_HUMAN",
"SPRC_HUMAN",
"CLCA_HUMAN",
"CLCB_HUMAN",
"ROA1_HUMAN",
"ALDOC_HUMAN",
"FBSP1_HUMAN",
"ELFN1_HUMAN",
"SRG2C_HUMAN",
"SRG2B_HUMAN",
"AA3R_HUMAN",
"CALM1_HUMAN",
"CALM2_HUMAN",
"CALM3_HUMAN",
"RAP2A_HUMAN",
"PTPRF_HUMAN",
"KAP0_HUMAN",
"CLUS_HUMAN",
"HPLN1_HUMAN",
"LAMC1_HUMAN",
"MTAP2_HUMAN",
"HSP7C_HUMAN",
"GTR3_HUMAN",
"ACM1_HUMAN",
"ACHB_HUMAN",
"RALA_HUMAN",
"BCR_HUMAN",
"SPTB1_HUMAN",
"LAMP1_HUMAN",
"FGFR1_HUMAN",
"DMD_HUMAN",
"PABP1_HUMAN",
"NFH_HUMAN",
"ACTN1_HUMAN",
"CADH1_HUMAN",
"SRC_HUMAN",
"SCG2_HUMAN",
"NCAM1_HUMAN",
"CSPG2_HUMAN",
"AT1A3_HUMAN",
"EF2_HUMAN",
"PLST_HUMAN",
"KAP2_HUMAN",
"DRD2_HUMAN",
"AK1A1_HUMAN",
"GLUT4_HUMAN",
"MMP9_HUMAN",
"HNRPL_HUMAN",
"GBRA1_HUMAN",
"BRAF_HUMAN",
"EZRI_HUMAN",
"VATB1_HUMAN",
"RS2_HUMAN",
"CD44_HUMAN",
"PGCA_HUMAN",
"ITB4_HUMAN",
"GDC_HUMAN",
"PP2BB_HUMAN",
"KCNA2_HUMAN",
"YBOX3_HUMAN",
"KPCA_HUMAN",
"ITA2_HUMAN",
"SYN1_HUMAN",
"KAPCA_HUMAN",
"CAN2_HUMAN",
"NEUM_HUMAN",
"ACHB2_HUMAN",
"PTN1_HUMAN",
"RL35A_HUMAN",
"ITB5_HUMAN",
"ARF4_HUMAN",
"RL7_HUMAN",
"PTPRA_HUMAN",
"GBRB1_HUMAN",
"GBRG2_HUMAN",
"PACA_HUMAN",
"RL17_HUMAN",
"ADA2C_HUMAN",
"CADH2_HUMAN",
"GNAZ_HUMAN",
"PLCG1_HUMAN",
"AT2B1_HUMAN",
"IF2B_HUMAN",
"ACM3_HUMAN",
"RAB3A_HUMAN",
"RAB3B_HUMAN",
"RAB4A_HUMAN",
"RAB5A_HUMAN",
"RAB6A_HUMAN",
"TKN1_HUMAN",
"A1ATR_HUMAN",
"PTN_HUMAN",
"VATB2_HUMAN",
"VATC1_HUMAN",
"FLNA_HUMAN",
"NF1_HUMAN",
"S1PR1_HUMAN",
"CNR1_HUMAN",
"SYT1_HUMAN",
"DRD1_HUMAN",
"FGF7_HUMAN",
"VDAC1_HUMAN",
"DRD4_HUMAN",
"DRD5_HUMAN",
"COMT_HUMAN",
"KCNA3_HUMAN",
"ACES_HUMAN",
"KCNA4_HUMAN",
"ROA2_HUMAN",
"CALB2_HUMAN",
"SFPQ_HUMAN",
"RS3_HUMAN",
"GLRA1_HUMAN",
"GLRA2_HUMAN",
"CBLN1_HUMAN",
"KS6B1_HUMAN",
"PTPRD_HUMAN",
"PTPRZ_HUMAN",
"COF1_HUMAN",
"BDNF_HUMAN",
"IF4B_HUMAN",
"AT2B4_HUMAN",
"IP3KA_HUMAN",
"VAMP1_HUMAN",
"SC6A2_HUMAN",
"LAMA2_HUMAN",
"GBRR1_HUMAN",
"AKAP5_HUMAN",
"TENA_HUMAN",
"APC_HUMAN",
"ARBK1_HUMAN",
"ADA1D_HUMAN",
"NK1R_HUMAN",
"RS12_HUMAN",
"DNJB1_HUMAN",
"NPY1R_HUMAN",
"ITA3_HUMAN",
"CTNA2_HUMAN",
"RL13_HUMAN",
"ELAV4_HUMAN",
"1433T_HUMAN",
"MK03_HUMAN",
"VATL_HUMAN",
"RL10_HUMAN",
"APEX1_HUMAN",
"CALR_HUMAN",
"PDE4A_HUMAN",
"CALX_HUMAN",
"5HT1D_HUMAN",
"5HT1B_HUMAN",
"5HT2A_HUMAN",
"GBRB3_HUMAN",
"GBRR2_HUMAN",
"MK01_HUMAN",
"GRN_HUMAN",
"AA2AR_HUMAN",
"AA2BR_HUMAN",
"EPHA3_HUMAN",
"EPHB2_HUMAN",
"NOS1_HUMAN",
"EF1D_HUMAN",
"MARCS_HUMAN",
"GNA11_HUMAN",
"RL12_HUMAN",
"2AAA_HUMAN",
"2AAB_HUMAN",
"SC6A1_HUMAN",
"ACHA5_HUMAN",
"AA1R_HUMAN",
"ACHB4_HUMAN",
"COR1A_HUMAN",
"GDIA_HUMAN",
"KAP1_HUMAN",
"KAP3_HUMAN",
"SC6A6_HUMAN",
"GBRA5_HUMAN",
"SC6A4_HUMAN",
"AKT1_HUMAN",
"HNRH1_HUMAN",
"L1CAM_HUMAN",
"ARRB2_HUMAN",
"ACHA3_HUMAN",
"NAC1_HUMAN",
"STX2_HUMAN",
"RL9_HUMAN",
"KINH_HUMAN",
"SDC2_HUMAN",
"GBRA3_HUMAN",
"RYK_HUMAN",
"5HT7R_HUMAN",
"CNR2_HUMAN",
"CRFR1_HUMAN",
"GPC1_HUMAN",
"PROF2_HUMAN",
"UBP6_HUMAN",
"CTNB1_HUMAN",
"PHB1_HUMAN",
"RADI_HUMAN",
"RL22_HUMAN",
"ADA1A_HUMAN",
"OPRM_HUMAN",
"DRD3_HUMAN",
"SCN1A_HUMAN",
"MYH10_HUMAN",
"ACTN2_HUMAN",
"ADDA_HUMAN",
"ADDB_HUMAN",
"ARBK2_HUMAN",
"FUS_HUMAN",
"VATE1_HUMAN",
"ACHA7_HUMAN",
"RL4_HUMAN",
"PP1G_HUMAN",
"SYUA_HUMAN",
"RBMX_HUMAN",
"GNAL_HUMAN",
"VATA_HUMAN",
"IF4A3_HUMAN",
"RS19_HUMAN",
"RL3_HUMAN",
"GRIK1_HUMAN",
"CAP2_HUMAN",
"ADCY8_HUMAN",
"VHL_HUMAN",
"RL13A_HUMAN",
"STAT3_HUMAN",
"UBP8_HUMAN",
"IF2G_HUMAN",
"OPRD_HUMAN",
"OPRK_HUMAN",
"CASR_HUMAN",
"OX2G_HUMAN",
"WNT5A_HUMAN",
"P2RY2_HUMAN",
"S19A1_HUMAN",
"GRM5_HUMAN",
"TSN7_HUMAN",
"KPCI_HUMAN",
"GRIA1_HUMAN",
"GRIA2_HUMAN",
"GRIA3_HUMAN",
"MTOR_HUMAN",
"EPS15_HUMAN",
"CASP3_HUMAN",
"DPP6_HUMAN",
"RS27_HUMAN",
"ABL2_HUMAN",
"RL35_HUMAN",
"NSG1_HUMAN",
"HD_HUMAN",
"EAA2_HUMAN",
"EAA3_HUMAN",
"LIS1_HUMAN",
"DCC_HUMAN",
"ACHA4_HUMAN",
"UBP5_HUMAN",
"MK08_HUMAN",
"MK09_HUMAN",
"5HT3A_HUMAN",
"CRK_HUMAN",
"CRKL_HUMAN",
"NSF_HUMAN",
"NOTC1_HUMAN",
"RL27A_HUMAN",
"RL5_HUMAN",
"RL21_HUMAN",
"RL28_HUMAN",
"RS9_HUMAN",
"RS5_HUMAN",
"RS10_HUMAN",
"MAP1B_HUMAN",
"NEDD4_HUMAN",
"IQGA1_HUMAN",
"CAPZB_HUMAN",
"IF1AX_HUMAN",
"GBRA2_HUMAN",
"GBRB2_HUMAN",
"5HT5A_HUMAN",
"P2RY1_HUMAN",
"RL29_HUMAN",
"NPTX2_HUMAN",
"SC6A8_HUMAN",
"KCNJ4_HUMAN",
"KCNJ6_HUMAN",
"GRIA4_HUMAN",
"S6A11_HUMAN",
"SC6A9_HUMAN",
"GLRB_HUMAN",
"GBRA4_HUMAN",
"PP2BC_HUMAN",
"KCNC1_HUMAN",
"KCNJ3_HUMAN",
"EAA4_HUMAN",
"TRPC1_HUMAN",
"MRP_HUMAN",
"NPY2R_HUMAN",
"RL34_HUMAN",
"LMAN1_HUMAN",
"ARRB1_HUMAN",
"EFTU_HUMAN",
"AMPH_HUMAN",
"NUMB_HUMAN",
"PSN1_HUMAN",
"EI2BB_HUMAN",
"RGS4_HUMAN",
"RGS7_HUMAN",
"PSN2_HUMAN",
"TSC2_HUMAN",
"GSK3B_HUMAN",
"GNAQ_HUMAN",
"GBG4_HUMAN",
"GDIB_HUMAN",
"PDLI4_HUMAN",
"VASP_HUMAN",
"DYN2_HUMAN",
"PPT1_HUMAN",
"RL14_HUMAN",
"FXR1_HUMAN",
"FXR2_HUMAN",
"RAB5C_HUMAN",
"RAB7A_HUMAN",
"P2RX1_HUMAN",
"P2RY4_HUMAN",
"MECP2_HUMAN",
"GPM6A_HUMAN",
"CLCN3_HUMAN",
"VAMP7_HUMAN",
"ROA3_HUMAN",
"HNRPM_HUMAN",
"IMA1_HUMAN",
"IMA5_HUMAN",
"DGKE_HUMAN",
"GDIR1_HUMAN",
"HNRPF_HUMAN",
"EFNB2_HUMAN",
"EFNA5_HUMAN",
"DGKQ_HUMAN",
"DAPK1_HUMAN",
"LIMK1_HUMAN",
"AP3M2_HUMAN",
"AP2S1_HUMAN",
"ITA8_HUMAN",
"MK10_HUMAN",
"MOT1_HUMAN",
"VMAT1_HUMAN",
"HAP1_HUMAN",
"CA2D1_HUMAN",
"UBP14_HUMAN",
"EPHB1_HUMAN",
"EPHA4_HUMAN",
"DVLP1_HUMAN",
"SNAA_HUMAN",
"IF5_HUMAN",
"TERA_HUMAN",
"AFAD_HUMAN",
"LAMB2_HUMAN",
"CADH6_HUMAN",
"CADH8_HUMAN",
"CAD11_HUMAN",
"CAD13_HUMAN",
"TPD52_HUMAN",
"HNRH2_HUMAN",
"SUMO3_HUMAN",
"EIF3B_HUMAN",
"P2RX3_HUMAN",
"AP1S2_HUMAN",
"IF6_HUMAN",
"CTBP2_HUMAN",
"WNT3A_HUMAN",
"NETR_HUMAN",
"BACE1_HUMAN",
"NRG3_HUMAN",
"NAC3_HUMAN",
"SNX16_HUMAN",
"NRX1B_HUMAN",
"NRX2B_HUMAN",
"NXPH1_HUMAN",
"GBG2_HUMAN",
"AP3S2_HUMAN",
"EIF3E_HUMAN",
"PTEN_HUMAN",
"ACTB_HUMAN",
"RAC3_HUMAN",
"RS20_HUMAN",
"SNP25_HUMAN",
"CDC42_HUMAN",
"DEST_HUMAN",
"RAB8A_HUMAN",
"RAB4B_HUMAN",
"RAB2A_HUMAN",
"RAB5B_HUMAN",
"RAB10_HUMAN",
"UBC12_HUMAN",
"UBE2N_HUMAN",
"RAB14_HUMAN",
"ARP3_HUMAN",
"ARP2_HUMAN",
"RAP1B_HUMAN",
"RS3A_HUMAN",
"RL26_HUMAN",
"STX1B_HUMAN",
"SMS_HUMAN",
"RL15_HUMAN",
"RL27_HUMAN",
"VA0D1_HUMAN",
"RL37A_HUMAN",
"RHOA_HUMAN",
"STXB1_HUMAN",
"RL37_HUMAN",
"SUMO2_HUMAN",
"AP1S1_HUMAN",
"HNRPK_HUMAN",
"1433G_HUMAN",
"UBP46_HUMAN",
"RS7_HUMAN",
"PP1A_HUMAN",
"NCS1_HUMAN",
"RS8_HUMAN",
"RS15A_HUMAN",
"RS16_HUMAN",
"1433E_HUMAN",
"RS14_HUMAN",
"RS23_HUMAN",
"RS18_HUMAN",
"RS13_HUMAN",
"RS11_HUMAN",
"ARF6_HUMAN",
"RL7A_HUMAN",
"RB11A_HUMAN",
"RS4X_HUMAN",
"RHOB_HUMAN",
"RL23A_HUMAN",
"RS6_HUMAN",
"RAB1A_HUMAN",
"RL23_HUMAN",
"RAP1A_HUMAN",
"RS15_HUMAN",
"RS24_HUMAN",
"RS25_HUMAN",
"RS26_HUMAN",
"RS28_HUMAN",
"GBB1_HUMAN",
"GBB2_HUMAN",
"RL30_HUMAN",
"RL31_HUMAN",
"RL10A_HUMAN",
"RL32_HUMAN",
"RL11_HUMAN",
"RL8_HUMAN",
"FKB1A_HUMAN",
"CCG7_HUMAN",
"RS27A_HUMAN",
"RL40_HUMAN",
"GRB2_HUMAN",
"RAC1_HUMAN",
"AP2B1_HUMAN",
"VAMP2_HUMAN",
"GNAS2_HUMAN",
"GNAI1_HUMAN",
"CANB1_HUMAN",
"1433Z_HUMAN",
"2ABA_HUMAN",
"SUMO1_HUMAN",
"RL38_HUMAN",
"SKP1_HUMAN",
"GBG3_HUMAN",
"RS21_HUMAN",
"IF5A1_HUMAN",
"KCNJ2_HUMAN",
"ACTG_HUMAN",
"UBC9_HUMAN",
"PP2AA_HUMAN",
"YBOX1_HUMAN",
"CSK2B_HUMAN",
"ACTC_HUMAN",
"EF1A1_HUMAN",
"CXAR_HUMAN",
"GBRE_HUMAN",
"IF4G2_HUMAN",
"ASIC1_HUMAN",
"DLG4_HUMAN",
"CNTP1_HUMAN",
"SRPK2_HUMAN",
"RELN_HUMAN",
"BL1S1_HUMAN",
"MAP1A_HUMAN",
"GPSM2_HUMAN",
"RL24_HUMAN",
"RL36A_HUMAN",
"HPCA_HUMAN",
"ARF1_HUMAN",
"RHOG_HUMAN",
"RL19_HUMAN",
"ALEX_HUMAN",
"VLDLR_HUMAN",
"EFNB1_HUMAN",
"EM55_HUMAN",
"CDK5_HUMAN",
"CDK16_HUMAN",
"PURA_HUMAN",
"CLH1_HUMAN",
"PLCB2_HUMAN",
"HNRPU_HUMAN",
"CAC1B_HUMAN",
"MDM2_HUMAN",
"SPTB2_HUMAN",
"RELB_HUMAN",
"FABP5_HUMAN",
"CAP1_HUMAN",
"CAC1D_HUMAN",
"AT2B2_HUMAN",
"SC6A3_HUMAN",
"PLCB3_HUMAN",
"ROR2_HUMAN",
"GCYA1_HUMAN",
"GCYB1_HUMAN",
"KPCE_HUMAN",
"CNTN2_HUMAN",
"NRG1_HUMAN",
"APBA1_HUMAN",
"RL18A_HUMAN",
"CACB1_HUMAN",
"MP2K1_HUMAN",
"RL6_HUMAN",
"AKA12_HUMAN",
"KCNC4_HUMAN",
"TF65_HUMAN",
"IF4G1_HUMAN",
"NOTC2_HUMAN",
"ACHE_HUMAN",
"1433F_HUMAN",
"UBE3A_HUMAN",
"DYN1_HUMAN",
"DCE2_HUMAN",
"FAK1_HUMAN",
"KPCZ_HUMAN",
"NMDZ1_HUMAN",
"EF1A2_HUMAN",
"KPCD_HUMAN",
"CALD1_HUMAN",
"ACHB3_HUMAN",
"VMAT2_HUMAN",
"APLP2_HUMAN",
"FMR1_HUMAN",
"ACHD_HUMAN",
"RL18_HUMAN",
"C1QBP_HUMAN",
"B2CL1_HUMAN",
"SOS1_HUMAN",
"ACK1_HUMAN",
"LRP1_HUMAN",
"FOXM1_HUMAN",
"PP2BA_HUMAN",
"CACB2_HUMAN",
"DEMA_HUMAN",
"TICN1_HUMAN",
"CD47_HUMAN",
"ADCY1_HUMAN",
"VAC14_HUMAN",
"ABCC8_HUMAN",
"KCNA1_HUMAN",
"FCHO2_HUMAN",
"AP1B1_HUMAN",
"KIF1A_HUMAN",
"SCRN1_HUMAN",
"KCMA1_HUMAN",
"KIF5A_HUMAN",
"STX4_HUMAN",
"CNTN1_HUMAN",
"NMDE1_HUMAN",
"ELAV2_HUMAN",
"EPS8_HUMAN",
"ANK3_HUMAN",
"DLG1_HUMAN",
"ABR_HUMAN",
"GRIK2_HUMAN",
"GRIK3_HUMAN",
"TIAM1_HUMAN",
"STRN3_HUMAN",
"LIPA1_HUMAN",
"ROA0_HUMAN",
"PAK1_HUMAN",
"PAK2_HUMAN",
"CBLB_HUMAN",
"NMDE2_HUMAN",
"GRM1_HUMAN",
"SEM3F_HUMAN",
"STX3_HUMAN",
"KCAB2_HUMAN",
"ATM_HUMAN",
"CD5R2_HUMAN",
"PTPRS_HUMAN",
"EIF3I_HUMAN",
"CTBP1_HUMAN",
"AP3B2_HUMAN",
"MPP3_HUMAN",
"PLD1_HUMAN",
"UB2V1_HUMAN",
"LSAMP_HUMAN",
"MYO9B_HUMAN",
"ROCK1_HUMAN",
"FZD5_HUMAN",
"DRP2_HUMAN",
"PICAL_HUMAN",
"SQSTM_HUMAN",
"PIN1_HUMAN",
"ATR_HUMAN",
"4EBP1_HUMAN",
"4EBP2_HUMAN",
"KCC2B_HUMAN",
"KCC2G_HUMAN",
"KCC2D_HUMAN",
"ULA1_HUMAN",
"DGKZ_HUMAN",
"SNX1_HUMAN",
"CUL3_HUMAN",
"PTC1_HUMAN",
"5HT4R_HUMAN",
"RAPSN_HUMAN",
"SPTN1_HUMAN",
"CAC1C_HUMAN",
"RGRF1_HUMAN",
"KCNC3_HUMAN",
"KCC1A_HUMAN",
"HNRPD_HUMAN",
"LRP8_HUMAN",
"DAG1_HUMAN",
"SEPT6_HUMAN",
"EIF3A_HUMAN",
"ARHG7_HUMAN",
"EFR3A_HUMAN",
"SCRIB_HUMAN",
"GIT2_HUMAN",
"MPP2_HUMAN",
"DOC2A_HUMAN",
"DOC2B_HUMAN",
"DOCK1_HUMAN",
"DPYL1_HUMAN",
"DPYL3_HUMAN",
"SRC8_HUMAN",
"FLOT2_HUMAN",
"FAK2_HUMAN",
"GNA13_HUMAN",
"GRM2_HUMAN",
"CAPR1_HUMAN",
"SPRL1_HUMAN",
"SEM3A_HUMAN",
"ITPR3_HUMAN",
"ITPR1_HUMAN",
"KCJ11_HUMAN",
"DIP2A_HUMAN",
"KCNB1_HUMAN",
"GRM7_HUMAN",
"GRM3_HUMAN",
"GRM4_HUMAN",
"CHD4_HUMAN",
"LASP1_HUMAN",
"NMDE3_HUMAN",
"OPCM_HUMAN",
"FRPD4_HUMAN",
"SEPT2_HUMAN",
"EEA1_HUMAN",
"CD5R1_HUMAN",
"PLCB4_HUMAN",
"STK38_HUMAN",
"NECT1_HUMAN",
"RABE1_HUMAN",
"RAB35_HUMAN",
"ERBB4_HUMAN",
"RBP1_HUMAN",
"L2GL1_HUMAN",
"PCBP1_HUMAN",
"PCBP2_HUMAN",
"EPHA7_HUMAN",
"RHEB_HUMAN",
"CNN3_HUMAN",
"KS6A1_HUMAN",
"CYH1_HUMAN",
"SHH_HUMAN",
"TERF2_HUMAN",
"DIXC1_HUMAN",
"DLG2_HUMAN",
"ELAV1_HUMAN",
"MYLK_HUMAN",
"NPY5R_HUMAN",
"EFNB3_HUMAN",
"MLF2_HUMAN",
"ITSN1_HUMAN",
"NPTX1_HUMAN",
"ACHA6_HUMAN",
"VAMP3_HUMAN",
"KCNJ8_HUMAN",
"NEDD8_HUMAN",
"CAC1E_HUMAN",
"VAS1_HUMAN",
"RB11B_HUMAN",
"GRIK4_HUMAN",
"SYUB_HUMAN",
"SEPT7_HUMAN",
"NTRK3_HUMAN",
"AINX_HUMAN",
"LAMA4_HUMAN",
"GBRA6_HUMAN",
"GRIK5_HUMAN",
"PKN1_HUMAN",
"ASIC2_HUMAN",
"MK14_HUMAN",
"DPYL2_HUMAN",
"SYPL1_HUMAN",
"KCC4_HUMAN",
"VACHT_HUMAN",
"CALRL_HUMAN",
"NTRK2_HUMAN",
"STX1A_HUMAN",
"DREB_HUMAN",
"AT2B3_HUMAN",
"PTPRO_HUMAN",
"PTPRN_HUMAN",
"VATF_HUMAN",
"RHG44_HUMAN",
"CPEB4_HUMAN",
"HNRL2_HUMAN",
"BLTP1_HUMAN",
"AAK1_HUMAN",
"INSY1_HUMAN",
"VWC2_HUMAN",
"S4A8_HUMAN",
"LEGL_HUMAN",
"SV2C_HUMAN",
"VP26B_HUMAN",
"ANO6_HUMAN",
"GRAP1_HUMAN",
"AMOT_HUMAN",
"ACTBL_HUMAN",
"IGS11_HUMAN",
"IQEC2_HUMAN",
"KCNT1_HUMAN",
"HABP4_HUMAN",
"GNAS1_HUMAN",
"PPR29_HUMAN",
"LRRK2_HUMAN",
"SNP47_HUMAN",
"TM240_HUMAN",
"TPRGL_HUMAN",
"STXB5_HUMAN",
"SYT6_HUMAN",
"MGLYR_HUMAN",
"WLS_HUMAN",
"MAGI3_HUMAN",
"AB17B_HUMAN",
"SYDE2_HUMAN",
"RN220_HUMAN",
"C1QL3_HUMAN",
"SUSD4_HUMAN",
"BTBD8_HUMAN",
"ANO1_HUMAN",
"PLCX3_HUMAN",
"A16L1_HUMAN",
"KCD16_HUMAN",
"MCTP2_HUMAN",
"MCTP1_HUMAN",
"IQEC1_HUMAN",
"NAPEP_HUMAN",
"R7BP_HUMAN",
"TEN4_HUMAN",
"C2D1A_HUMAN",
"F171B_HUMAN",
"CNKR3_HUMAN",
"AB17C_HUMAN",
"CNIH2_HUMAN",
"GLD2_HUMAN",
"LRFN4_HUMAN",
"CPLX2_HUMAN",
"SARM1_HUMAN",
"KLH17_HUMAN",
"CRTC1_HUMAN",
"CBLN3_HUMAN",
"TM108_HUMAN",
"GSG1L_HUMAN",
"NSMF_HUMAN",
"SYT10_HUMAN",
"DNMBP_HUMAN",
"INSY2_HUMAN",
"SHSA6_HUMAN",
"SYDE1_HUMAN",
"KCTD8_HUMAN",
"RAPH1_HUMAN",
"TBA1A_HUMAN",
"MARK2_HUMAN",
"SV2A_HUMAN",
"SV2B_HUMAN",
"CYFP1_HUMAN",
"FA2H_HUMAN",
"LIGO2_HUMAN",
"ARC_HUMAN",
"PLPR4_HUMAN",
"NEGR1_HUMAN",
"ATG9A_HUMAN",
"PRIC2_HUMAN",
"UBE3B_HUMAN",
"CSMD2_HUMAN",
"KI21A_HUMAN",
"MDGA2_HUMAN",
"RHG22_HUMAN",
"C1QL2_HUMAN",
"CPEB2_HUMAN",
"NECA2_HUMAN",
"ANS1B_HUMAN",
"GRASP_HUMAN",
"PRRT2_HUMAN",
"FLIP1_HUMAN",
"LHPL4_HUMAN",
"AGRA1_HUMAN",
"SYT9_HUMAN",
"LRRT1_HUMAN",
"MAGI2_HUMAN",
"LHPL3_HUMAN",
"RIMS1_HUMAN",
"CAPS2_HUMAN",
"LRRT4_HUMAN",
"LRRT3_HUMAN",
"KCMB4_HUMAN",
"STX12_HUMAN",
"HOME1_HUMAN",
"MIB1_HUMAN",
"RB6I2_HUMAN",
"ZDH17_HUMAN",
"CBLN2_HUMAN",
"SYT12_HUMAN",
"SLIK4_HUMAN",
"RHG12_HUMAN",
"KIBRA_HUMAN",
"SIR2_HUMAN",
"ABI1_HUMAN",
"CA2D3_HUMAN",
"NLGNX_HUMAN",
"CADM3_HUMAN",
"VPS52_HUMAN",
"DOCK4_HUMAN",
"NLGN1_HUMAN",
"ELMD1_HUMAN",
"CADM2_HUMAN",
"SYNPO_HUMAN",
"PNKD_HUMAN",
"MINK1_HUMAN",
"SVOP_HUMAN",
"RNF10_HUMAN",
"NGEF_HUMAN",
"ARFG1_HUMAN",
"ENAH_HUMAN",
"SYT2_HUMAN",
"TDRD5_HUMAN",
"UN13C_HUMAN",
"ATAD1_HUMAN",
"NETO2_HUMAN",
"NECP1_HUMAN",
"CAMKV_HUMAN",
"CARL3_HUMAN",
"LIPB2_HUMAN",
"VGLU3_HUMAN",
"CPEB3_HUMAN",
"PK3C3_HUMAN",
"NGDN_HUMAN",
"TRPV1_HUMAN",
"SYNE1_HUMAN",
"MDGA1_HUMAN",
"NBEA_HUMAN",
"NLGNY_HUMAN",
"NLGN2_HUMAN",
"CADM4_HUMAN",
"VA0E2_HUMAN",
"SCAM5_HUMAN",
"PRR7_HUMAN",
"PUM2_HUMAN",
"LNX1_HUMAN",
"CNIH3_HUMAN",
"FA81A_HUMAN",
"SYNPR_HUMAN",
"TM163_HUMAN",
"CPT1C_HUMAN",
"NMD3A_HUMAN",
"BRSK1_HUMAN",
"NETO1_HUMAN",
"DMXL2_HUMAN",
"DLG5_HUMAN",
"GP151_HUMAN",
"PRUN2_HUMAN",
"RN138_HUMAN",
"CPLX3_HUMAN",
"HNRLL_HUMAN",
"CSKI1_HUMAN",
"STON2_HUMAN",
"MADD_HUMAN",
"CNKR2_HUMAN",
"CCG8_HUMAN",
"NANO1_HUMAN",
"DGCR8_HUMAN",
"CTTB2_HUMAN",
"RPGF4_HUMAN",
"NICA_HUMAN",
"ELMO1_HUMAN",
"WASF1_HUMAN",
"AP3S1_HUMAN",
"TSC1_HUMAN",
"SL9A6_HUMAN",
"SEPT8_HUMAN",
"LPAR1_HUMAN",
"AKAP1_HUMAN",
"NEUG_HUMAN",
"PTPRU_HUMAN",
"ARC1A_HUMAN",
"TENR_HUMAN",
"SYN2_HUMAN",
"DLG3_HUMAN",
"KCNJ9_HUMAN",
"NRCAM_HUMAN",
"NELL1_HUMAN",
"KIFA3_HUMAN",
"GHSR_HUMAN",
"SEM4D_HUMAN",
"NEO1_HUMAN",
"RAB8B_HUMAN",
"PTPR2_HUMAN",
"ARHG2_HUMAN",
"DVL3_HUMAN",
"USP9X_HUMAN",
"VPP1_HUMAN",
"RNF34_HUMAN",
"MAL2_HUMAN",
"IGSF8_HUMAN",
"MRTFA_HUMAN",
"CPNE4_HUMAN",
"TM230_HUMAN",
"EXOC4_HUMAN",
"FUBP1_HUMAN",
"VTI1A_HUMAN",
"MIB2_HUMAN",
"SH3K1_HUMAN",
"APH1A_HUMAN",
"ARL8A_HUMAN",
"DOC10_HUMAN",
"EVI5L_HUMAN",
"AP2M1_HUMAN",
"NTNG2_HUMAN",
"KCD12_HUMAN",
"LEG12_HUMAN",
"RAB3C_HUMAN",
"NTCP4_HUMAN",
"DTBP1_HUMAN",
"CYFP2_HUMAN",
"CNRP1_HUMAN",
"LIGO1_HUMAN",
"DYL2_HUMAN",
"PLPP_HUMAN",
"AB17A_HUMAN",
"PGCB_HUMAN",
"PDLI5_HUMAN",
"CREL1_HUMAN",
"SENP5_HUMAN",
"IGS21_HUMAN",
"FXL20_HUMAN",
"VCIP1_HUMAN",
"EXOC2_HUMAN",
"SNX27_HUMAN",
"TRI47_HUMAN",
"TAFA4_HUMAN",
"CI072_HUMAN",
"PRIC1_HUMAN",
"DOCK7_HUMAN",
"LRFN5_HUMAN",
"LRRC7_HUMAN",
"AGAP3_HUMAN",
"INP4A_HUMAN",
"KCNC2_HUMAN",
"NED4L_HUMAN",
"QKI_HUMAN",
"SYGP1_HUMAN",
"SLIK1_HUMAN",
"FMNL2_HUMAN",
"BTBD9_HUMAN",
"ALS2_HUMAN",
"VPS35_HUMAN",
"MAGI1_HUMAN",
"ERBIN_HUMAN",
"KS6C1_HUMAN",
"NEB2_HUMAN",
"RMD3_HUMAN",
"SCN2A_HUMAN",
"DCE1_HUMAN",
"CYH2_HUMAN",
"NELL2_HUMAN",
"AGAP2_HUMAN",
"PARK7_HUMAN",
"S1PR3_HUMAN",
"SORT_HUMAN",
"GPER1_HUMAN",
"P2RX4_HUMAN",
"P2RX7_HUMAN",
"PHB2_HUMAN",
"KIF2C_HUMAN",
"MGLL_HUMAN",
"FEZ1_HUMAN",
"SEPT5_HUMAN",
"SGMR1_HUMAN",
"ZNT3_HUMAN",
"ROAA_HUMAN",
"SNAG_HUMAN",
"APBA2_HUMAN",
"NOE1_HUMAN",
"S29A1_HUMAN",
"SMO_HUMAN",
"SC6A7_HUMAN",
"GBRG3_HUMAN",
"PRRT1_HUMAN",
"SH3G1_HUMAN",
"SH3G2_HUMAN",
"SH3G3_HUMAN",
"AKAP9_HUMAN",
"DPYL5_HUMAN",
"ARP5L_HUMAN",
"SENP7_HUMAN",
"SYT3_HUMAN",
"SGIP1_HUMAN",
"PSD2_HUMAN",
"CSTN3_HUMAN",
"COR1B_HUMAN",
"SYT17_HUMAN",
"SYT11_HUMAN",
"LRFN3_HUMAN",
"AN32E_HUMAN",
"P4K2A_HUMAN",
"BEGIN_HUMAN",
"HNRL1_HUMAN",
"ABHD6_HUMAN",
"SCMC3_HUMAN",
"TBB2B_HUMAN",
"TMUB1_HUMAN",
"RFIP5_HUMAN",
"AP1M1_HUMAN",
"TDRD1_HUMAN",
"PACN1_HUMAN",
"CADM1_HUMAN",
"SHAN3_HUMAN",
"SE6L1_HUMAN",
"TBD2A_HUMAN",
"CPEB1_HUMAN",
"PPAT_HUMAN",
"RTN4R_HUMAN",
"TRIM9_HUMAN",
"CNTP4_HUMAN",
"ZDHC5_HUMAN",
"SEM4C_HUMAN",
"PHAR1_HUMAN",
"TANC1_HUMAN",
"GRIP2_HUMAN",
"RHG39_HUMAN",
"SRCN1_HUMAN",
"ROGDI_HUMAN",
"MLP3B_HUMAN",
"SENP6_HUMAN",
"EGLN1_HUMAN",
"ACH10_HUMAN",
"RB33B_HUMAN",
"SHRPN_HUMAN",
"FXYD6_HUMAN",
"RAB17_HUMAN",
"RAB1B_HUMAN",
"SNAB_HUMAN",
"SLIK2_HUMAN",
"BC11A_HUMAN",
"OSBL2_HUMAN",
"S6A17_HUMAN",
"ATG5_HUMAN",
"S1PR5_HUMAN",
"PORCN_HUMAN",
"CAD23_HUMAN",
"VP33B_HUMAN",
"VPS16_HUMAN",
"VPS11_HUMAN",
"SYT4_HUMAN",
"KCNN2_HUMAN",
"IPYR2_HUMAN",
"S12A5_HUMAN",
"TTYH1_HUMAN",
"PTN23_HUMAN",
"DNJC5_HUMAN",
"RIMS4_HUMAN",
"CHM4B_HUMAN",
"FZD8_HUMAN",
"MLP3A_HUMAN",
"WNK1_HUMAN",
"CSTN2_HUMAN",
"E41L1_HUMAN",
"EHD1_HUMAN",
"VIAAT_HUMAN",
"CLMP_HUMAN",
"ANTR1_HUMAN",
"SYNG1_HUMAN",
"CNNM2_HUMAN",
"LIN7B_HUMAN",
"AGRL3_HUMAN",
"PKHA5_HUMAN",
"GBB4_HUMAN",
"VPP4_HUMAN",
"VEZA_HUMAN",
"LPAR2_HUMAN",
"LRRC4_HUMAN",
"OTOF_HUMAN",
"PCDH9_HUMAN",
"TANC2_HUMAN",
"LRC4C_HUMAN",
"ROBO2_HUMAN",
"PLXA4_HUMAN",
"FGF22_HUMAN",
"NRX3B_HUMAN",
"IRPL2_HUMAN",
"ARFG3_HUMAN",
"RAB18_HUMAN",
"NRN1_HUMAN",
"FZD3_HUMAN",
"IL1AP_HUMAN",
"SEM4B_HUMAN",
"PLCB1_HUMAN",
"CRAC1_HUMAN",
"ANO2_HUMAN",
"RTN4_HUMAN",
"CYLD_HUMAN",
"NECT3_HUMAN",
"PAK6_HUMAN",
"GEPH_HUMAN",
"ANM8_HUMAN",
"SHLB2_HUMAN",
"KCNQ5_HUMAN",
"S17A5_HUMAN",
"PICK1_HUMAN",
"DISC1_HUMAN",
"STRN4_HUMAN",
"RNC_HUMAN",
"RAB6B_HUMAN",
"VPS45_HUMAN",
"HOME2_HUMAN",
"HOME3_HUMAN",
"TEN2_HUMAN",
"LRC4B_HUMAN",
"SAC1_HUMAN",
"CBLN4_HUMAN",
"STAU2_HUMAN",
"LIN7C_HUMAN",
"RN19A_HUMAN",
"SEP11_HUMAN",
"ARL8B_HUMAN",
"RN216_HUMAN",
"SMOX_HUMAN",
"CA2D2_HUMAN",
"SPHK1_HUMAN",
"ABI2_HUMAN",
"DACT1_HUMAN",
"PSD3_HUMAN",
"CELR3_HUMAN",
"CALY_HUMAN",
"PLK2_HUMAN",
"PEN2_HUMAN",
"NLGN3_HUMAN",
"IRPL1_HUMAN",
"SPN90_HUMAN",
"FLRT3_HUMAN",
"CABP1_HUMAN",
"KCND2_HUMAN",
"CRIPT_HUMAN",
"ADA22_HUMAN",
"FRS1L_HUMAN",
"VAPA_HUMAN",
"MARK1_HUMAN",
"AKA7G_HUMAN",
"SENP1_HUMAN",
"NTRI_HUMAN",
"DLGP2_HUMAN",
"LRFN1_HUMAN",
"VPS18_HUMAN",
"TEN3_HUMAN",
"PAK5_HUMAN",
"ABI3_HUMAN",
"KIF17_HUMAN",
"PCD10_HUMAN",
"SI1L2_HUMAN",
"CPSF2_HUMAN",
"TUTLA_HUMAN",
"NRX2A_HUMAN",
"VGLU1_HUMAN",
"VGLU2_HUMAN",
"GBG13_HUMAN",
"NCDN_HUMAN",
"EP15R_HUMAN",
"GBG12_HUMAN",
"CPNE7_HUMAN",
"P2RX2_HUMAN",
"CCG4_HUMAN",
"VPS29_HUMAN",
"GABR1_HUMAN",
"GHRL_HUMAN",
"PPR1B_HUMAN",
"VTI1B_HUMAN",
"CCG5_HUMAN",
"ACHA9_HUMAN",
"SEPT3_HUMAN",
"SWP70_HUMAN",
"CNTP2_HUMAN",
"SEPT9_HUMAN",
"NUFP1_HUMAN",
"VATH_HUMAN",
"PRAF1_HUMAN",
"TAGL3_HUMAN",
"SCNBA_HUMAN",
"NCKX2_HUMAN",
"ZDHC2_HUMAN",
"PLXA1_HUMAN",
"RABX5_HUMAN",
"RIMS3_HUMAN",
"LAMP5_HUMAN",
"DBNL_HUMAN",
"PURG_HUMAN",
"ANC2_HUMAN",
"KCND3_HUMAN",
"FBX2_HUMAN",
"KS6A6_HUMAN",
"TNIK_HUMAN",
"DP13A_HUMAN",
"ADA28_HUMAN",
"AGO2_HUMAN",
"TEN1_HUMAN",
"RAB21_HUMAN",
"BL1S6_HUMAN",
"TAOK2_HUMAN",
"TRPC5_HUMAN",
"MKLN1_HUMAN",
"NRX1A_HUMAN",
"CADH9_HUMAN",
"ZDHC8_HUMAN",
"KDIS_HUMAN",
"ASAP1_HUMAN",
"LRFN2_HUMAN",
"MRTFB_HUMAN",
"NWD2_HUMAN",
"NEB1_HUMAN",
"GRID1_HUMAN",
"VANG2_HUMAN",
"SHRM4_HUMAN",
"TBC24_HUMAN",
"PPM1H_HUMAN",
"CAPS1_HUMAN",
"MYO5B_HUMAN",
"FZD4_HUMAN",
"RAB26_HUMAN",
"FZR1_HUMAN",
"MYO6_HUMAN",
"ALK_HUMAN",
"ICAM5_HUMAN",
"PACN2_HUMAN",
"SNX6_HUMAN",
"FZD1_HUMAN",
"BSN_HUMAN",
"IQEC3_HUMAN",
"AGAP1_HUMAN",
"NAC2_HUMAN",
"EXOC7_HUMAN",
"SORC3_HUMAN",
"TBD2B_HUMAN",
"SMAG1_HUMAN",
"PHF24_HUMAN",
"UN13A_HUMAN",
"TUTLB_HUMAN",
"SHAN2_HUMAN",
"DICER_HUMAN",
"WASF3_HUMAN",
"DYN3_HUMAN",
"RIMS2_HUMAN",
"CNTN6_HUMAN",
"CTND2_HUMAN",
"BAIP2_HUMAN",
"SCN8A_HUMAN",
"KCC2A_HUMAN",
"CHM2B_HUMAN",
"PDE10_HUMAN",
"LZTS1_HUMAN",
"EIF3L_HUMAN",
"TPPC4_HUMAN",
"NCKP1_HUMAN",
"DLGP4_HUMAN",
"ST38L_HUMAN",
"NTNG1_HUMAN",
"RP3A_HUMAN",
"E41L3_HUMAN",
"STB5L_HUMAN",
"AT8A1_HUMAN",
"AP3M1_HUMAN",
"GUAD_HUMAN",
"GIT1_HUMAN",
"NSG2_HUMAN",
"SC6A5_HUMAN",
"PHOCN_HUMAN",
"GRIP1_HUMAN",
"RL36_HUMAN",
"KIF3A_HUMAN",
"NRX3A_HUMAN",
"DAAM1_HUMAN",
"DGLA_HUMAN",
"PLXD1_HUMAN",
"WDR7_HUMAN",
"FARP1_HUMAN",
"TLN2_HUMAN",
"RPGF2_HUMAN",
"MYO5A_HUMAN",
"TRAF6_HUMAN",
"SHAN1_HUMAN",
"CD2AP_HUMAN",
"VATD_HUMAN",
"SNX14_HUMAN",
"SNX9_HUMAN",
"SCNAA_HUMAN",
"GPC6_HUMAN",
"NPTN_HUMAN",
"KPTN_HUMAN",
"CCG2_HUMAN",
"TACC3_HUMAN",
"BIG2_HUMAN",
"EPN1_HUMAN",
"ZNT1_HUMAN",
"CAD10_HUMAN",
"NCOA3_HUMAN",
"NUMBL_HUMAN",
"DGKB_HUMAN",
"PCLO_HUMAN",
"WASF2_HUMAN",
"PIAS3_HUMAN"

]

In [61]:
#Ubiquitin
sequences = [
# AFF4_HUMAN
"MNREDRNVLRMKERERRNQEIQQGEDAFPPSSPLFAEPYKVTSKEDKLSSRIQSMLGNYDEMKDFIGDRSIPKLVAIPKPTVPPSADEKSNPNFFEQRHGGSHQSSKWTPVGPAPSTSQSQKRSSGLQSGHSSQRTSAGSSSGTNSSGQRHDRESYNNSGSSSRKKGQHGSEHSKSRSSSPGKPQAVSSLNSSHSRSHGNDHHSKEHQRSKSPRDPDANWDSPSRVPFSSGQHSTQSFPPSLMSKSNSMLQKPTAYVRPMDGQESMEPKLSSEHYSSQSHGNSMTELKPSSKAHLTKLKIPSQPLDASASGDVSCVDEILKEMTHSWPPPLTAIHTPCKTEPSKFPFPTKESQQSNFGTGEQKRYNPSKTSNGHQSKSMLKDDLKLSSSEDSDGEQDCDKTMPRSTPGSNSEPSHHNSEGADNSRDDSSSHSGSESSSGSDSESESSSSDSEANEPSQSASPEPEPPPTNKWQLDNWLNKVNPHKVSPASSVDSNIPSSQGYKKEGREQGTGNSYTDTSGPKETSSATPGRDSKTIQKGSESGRGRQKSPAQSDSTTQRRTVGKKQPKKAEKAAAEEPRGGLKIESETPVDLASSMPSSRHKAATKGSRKPNIKKESKSSPRPTAEKKKYKSTSKSSQKSREIIETDTSSSDSDESESLPPSSQTPKYPESNRTPVKPSSVEEEDSFFRQRMFSPMEEKELLSPLSEPDDRYPLIVKIDLNLLTRIPGKPYKETEPPKGEKKNVPEKHTREAQKQASEKVSNKGKRKHKNEDDNRASESKKPKTEDKNSAGHKPSSNRESSKQSAAKEKDLLPSPAGPVPSKDPKTEHGSRKRTISQSSSLKSSSNSNKETSGSSKNSSSTSKQKKTEGKTSSSSKEVKEKAPSSSSNCPPSAPTLDSSKPRRTKLVFDDRNYSADHYLQEAKKLKHNADALSDRFEKAVYYLDAVVSFIECGNALEKNAQESKSPFPMYSETVDLIKYTMKLKNYLAPDATAADKRLTVLCLRCESLLYLRLFKLKKENALKYSKTLTEHLKNSYNNSQAPSPGLGSKAVGMPSPVSPKLSPGNSGNYSSGASSASASGSSVTIPQKIHQMAASYVQVTSNFLYATEIWDQAEQLSKEQKEFFAELDKVMGPLIFNASIMTDLVRYTRQGLHWLRQDAKLIS",
# AMFR_HUMAN
"MPLLFLERFPWPSLRTYTGLSGLALLGTIISAYRALSQPEAGPGEPDQLTASLQPEPPAPARPSAGGPRARDVAQYLLSDSLFVWVLVNTACCVLMLVAKLIQCIVFGPLRVSERQHLKDKFWNFIFYKFIFIFGVLNVQTVEEVVMWCLWFAGLVFLHLMVQLCKDRFEYLSFSPTTPMSSHGRVLSLLVAMLLSCCGLAAVCSITGYTHGMHTLAFMAAESLLVTVRTAHVILRYVIHLWDLNHEGTWEGKGTYVYYTDFVMELTLLSLDLMHHIHMLLFGNIWLSMASLVIFMQLRYLFHEVQRRIRRHKNYLRVVGNMEARFAVATPEELAVNNDDCAICWDSMQAARKLPCGHLFHNSCLRSWLEQDTSCPTCRMSLNIADNNRVREEHQGENLDENLVPVAAAEGRPRLNQHNHFFHFDGSRIASWLPSFSVEVMHTTNILGITQASNSQLNAMAHQIQEMFPQVPYHLVLQDLQLTRSVEITTDNILEGRIQVPFPTQRSDSIRPALNSPVERPSSDQEEGETSAQTERVPLDLSPRLEETLDFGEVEVEPSEVEDFEARGSRFSKSADERQRMLVQRKDELLQQARKRFLNKSSEDDAASESFLPSEGASSDPVTLRRRMLAAAAERRLQKQQTS",
# APC11_HUMAN
"MKVKIKCWNGVATWLWVANDENCGICRMAFNGCCPDCKVPGDDCPLVWGQCSHCFHMHCILKWLHAQQVQQHCPMCRQEWKFKE",
# AKIB1_HUMAN
"MGNTTTKFRKALINGDENLACQIYENNPQLKESLDPNTSYGEPYQHNTPLHYAARHGMNKILGTFLGRDGNPNKRNVHNETSMHLLCMGPQIMISEGALHPRLARPTEDDFRRADCLQMILKWKGAKLDQGEYERAAIDAVDNKKNTPLHYAAASGMKACVELLVKHGGDLFAENENKDTPCDCAEKQHHKDLALNLESQMVFSRDPEAEEIEAEYAALDKREPYEGLRPQDLRRLKDMLIVETADMLQAPLFTAEALLRAHDWDREKLLEAWMSNPENCCQRSGVQMPTPPPSGYNAWDTLPSPRTPRTTRSSVTSPDEISLSPGDLDTSLCDICMCSISVFEDPVDMPCGHDFCRGCWESFLNLKIQEGEAHNIFCPAYDCFQLVPVDIIESVVSKEMDKRYLQFDIKAFVENNPAIKWCPTPGCDRAVRLTKQGSNTSGSDTLSFPLLRAPAVDCGKGHLFCWECLGEAHEPCDCQTWKNWLQKITEMKPEELVGVSEAYEDAANCLWLLTNSKPCANCKSPIQKNEGCNHMQCAKCKYDFCWICLEEWKKHSSSTGGYYRCTRYEVIQHVEEQSKEMTVEAEKKHKRFQELDRFMHYYTRFKNHEHSYQLEQRLLKTAKEKMEQLSRALKETEGGCPDTTFIEDAVHVLLKTRRILKCSYPYGFFLEPKSTKKEIFELMQTDLEMVTEDLAQKVNRPYLRTPRHKIIKAACLVQQKRQEFLASVARGVAPADSPEAPRRSFAGGTWDWEYLGFASPEEYAEFQYRRRHRQRRRGDVHSLLSNPPDPDEPSESTLDIPEGGSSSRRPGTSVVSSASMSVLHSSSLRDYTPASRSENQDSLQALSSLDEDDPNILLAIQLSLQESGLALDEETRDFLSNEASLGAIGTSLPSRLDSVPRNTDSPRAALSSSELLELGDSLMRLGAENDPFSTDTLSSHPLSEARSDFCPSSSDPDSAGQDPNINDNLLGNIMAWFHDMNPQSIALIPPATTEISADSQLPCIKDGSEGVKDVELVLPEDSMFEDASVSEGRGTQIEENPLEENILAGEAASQAGDSGNEAANRGDGSDVSSQTPQTSSDWLEQVHLV",
# AREL1_HUMAN
"MFYVIGGITVSVVAFFFTIKFLFELAARVVSFLQNEDRERRGDRTIYDYVRGNYLDPRSCKVSWDWKDPYEVGHSMAFRVHLFYKNGQPFPAHRPVGLRVHISHVELAVEIPVTQEVLQEPNSNVVKVAFTVRKAGRYEITVKLGGLNVAYSPYYKIFQPGMVVPSKTKIVCHFSTLVLTCGQPHTLQIVPRDEYDNPTNNSMSLRDEHNYTLSIHELGPQEEESTGVSFEKSVTSNRQTFQVFLRLTLHSRGCFHACISYQNQPINNGEFDIIVLSEDEKNIVERNVSTSGVSIYFEAYLYNATNCSSTPWHLPPMHMTSSQRRPSTAVDEEDEDSPSECHTPEKVKKPKKVYCYVSPKQFSVKEFYLKIIPWRLYTFRVCPGTKFSYLGPDPVHKLLTLVVDDGIQPPVELSCKERNILAATFIRSLHKNIGGSETFQDKVNFFQRELRQVHMKRPHSKVTLKVSRHALLESSLKATRNFSISDWSKNFEVVFQDEEALDWGGPRREWFELICKALFDTTNQLFTRFSDNNQALVHPNPNRPAHLRLKMYEFAGRLVGKCLYESSLGGAYKQLVRARFTRSFLAQIIGLRMHYKYFETDDPEFYKSKVCFILNNDMSEMELVFAEEKYNKSGQLDKVVELMTGGAQTPVTNANKIFYLNLLAQYRLASQVKEEVEHFLKGLNELVPENLLAIFDENELELLMCGTGDISVSDFKAHAVVVGGSWHFREKVMRWFWTVVSSLTQEELARLLQFTTGSSQLPPGGFAALCPSFQIIAAPTHSTLPTAHTCFNQLCLPTYDSYEEVHRMLQLAISEGCEGFGML",
# ARI1_HUMAN
"MDSDEGYNYEFDEDEECSEEDSGAEEEEDEDDDEPDDDTLDLGEVELVEPGLGVGGERDGLLCGETGGGGGSALGPGGGGGGGGGGGGGGPGHEQEEDYRYEVLTAEQILQHMVECIREVNEVIQNPATITRILLSHFNWDKEKLMERYFDGNLEKLFAECHVINPSKKSRTRQMNTRSSAQDMPCQICYLNYPNSYFTGLECGHKFCMQCWSEYLTTKIMEEGMGQTISCPAHGCDILVDDNTVMRLITDSKVKLKYQHLITNSFVECNRLLKWCPAPDCHHVVKVQYPDAKPVRCKCGRQFCFNCGENWHDPVKCKWLKKWIKKCDDDSETSNWIAANTKECPKCHVTIEKDGGCNHMVCRNQNCKAEFCWVCLGPWEPHGSAWYNCNRYNEDDAKAARDAQERSRAALQRYLFYCNRYMNHMQSLRFEHKLYAQVKQKMEEMQQHNMSWIEVQFLKKAVDVLCQCRATLMYTYVFAFYLKKNNQSIIFENNQADLENATEVLSGYLERDISQDSLQDIKQKVQDKYRYCESRRRVLLQHVHEGYEKDLWEYIED",
# ARI2_HUMAN
"MSVDMNSQGSDSNEEDYDPNCEEEEEEEEDDPGDIEDYYVGVASDVEQQGADAFDPEEYQFTCLTYKESEGALNEHMTSLASVLKVSHSVAKLILVNFHWQVSEILDRYKSNSAQLLVEARVQPNPSKHVPTSHPPHHCAVCMQFVRKENLLSLACQHQFCRSCWEQHCSVLVKDGVGVGVSCMAQDCPLRTPEDFVFPLLPNEELREKYRRYLFRDYVESHYQLQLCPGADCPMVIRVQEPRARRVQCNRCNEVFCFKCRQMYHAPTDCATIRKWLTKCADDSETANYISAHTKDCPKCNICIEKNGGCNHMQCSKCKHDFCWMCLGDWKTHGSEYYECSRYKENPDIVNQSQQAQAREALKKYLFYFERWENHNKSLQLEAQTYQRIHEKIQERVMNNLGTWIDWQYLQNAAKLLAKCRYTLQYTYPYAYYMESGPRKKLFEYQQAQLEAEIENLSWKVERADSYDRGDLENQMHIAEQRRRTLLKDFHDT",
# BARD1_HUMAN
"MPDNRQPRNRQPRIRSGNEPRSAPAMEPDGRGAWAHSRAALDRLEKLLRCSRCTNILREPVCLGGCEHIFCSNCVSDCIGTGCPVCYTPAWIQDLKINRQLDSMIQLCSKLRNLLHDNELSDLKEDKPRKSLFNDAGNKKNSIKMWFSPRSKKVRYVVSKASVQTQPAIKKDASAQQDSYEFVSPSPPADVSERAKKASARSGKKQKKKTLAEINQKWNLEAEKEDGEFDSKEESKQKLVSFCSQPSVISSPQINGEIDLLASGSLTESECFGSLTEVSLPLAEQIESPDTKSRNEVVTPEKVCKNYLTSKKSLPLENNGKRGHHNRLSSPISKRCRTSILSTSGDFVKQTVPSENIPLPECSSPPSCKRKVGGTSGRKNSNMSDEFISLSPGTPPSTLSSSSYRRVMSSPSAMKLLPNMAVKRNHRGETLLHIASIKGDIPSVEYLLQNGSDPNVKDHAGWTPLHEACNHGHLKVVELLLQHKALVNTTGYQNDSPLHDAAKNGHVDIVKLLLSYGASRNAVNIFGLRPVDYTDDESMKSLLLLPEKNESSSASHCSVMNTGQRRDGPLVLIGSGLSSEQQKMLSELAVILKAKKYTEFDSTVTHVVVPGDAVQSTLKCMLGILNGCWILKFEWVKACLRRKVCEQEEKYEIPEGPRRSRLNREQLLPKLFDGCYFYLWGTFKHHPKDNLIKLVTAGGGQILSRKPKPDSDVTQTINTVAYHARPDSDQRFCTQYIIYEDLCNYHPERVRQGKVWKAPSSWFIDCVMSFELLPLDS",
# BFAR_HUMAN
"MEEPQKSYVNTMDLERDEPLKSTGPQISVSEFSCHCCYDILVNPTTLNCGHSFCRHCLALWWASSKKTECPECREKWEGFPKVSILLRDAIEKLFPDAIRLRFEDIQQNNDIVQSLAAFQKYGNDQIPLAPNTGRANQQMGGGFFSGVLTALTGVAVVLLVYHWSSRESEHDLLVHKAVAKWTAEEVVLWLEQLGPWASLYRERFLSERVNGRLLLTLTEEEFSKTPYTIENSSHRRAILMELERVKALGVKPPQNLWEYKAVNPGRSLFLLYALKSSPRLSLLYLYLFDYTDTFLPFIHTICPLQEDSSGEDIVTKLLDLKEPTWKQWREFLVKYSFLPYQLIAEFAWDWLEVHYWTSRFLIINAMLLSVLELFSFWRIWSRSELKTVPQRMWSHFWKVSTQGLFVAMFWPLIPQFVCNCLFYWALYFNPIINIDLVVKELRRLETQVL",
# BIRC2_HUMAN
"MHKTASQRLFPGPSYQNIKSIMEDSTILSDWTNSNKQKMKYDFSCELYRMSTYSTFPAGVPVSERSLARAGFYYTGVNDKVKCFCCGLMLDNWKLGDSPIQKHKQLYPSCSFIQNLVSASLGSTSKNTSPMRNSFAHSLSPTLEHSSLFSGSYSSLSPNPLNSRAVEDISSSRTNPYSYAMSTEEARFLTYHMWPLTFLSPSELARAGFYYIGPGDRVACFACGGKLSNWEPKDDAMSEHRRHFPNCPFLENSLETLRFSISNLSMQTHAARMRTFMYWPSSVPVQPEQLASAGFYYVGRNDDVKCFCCDGGLRCWESGDDPWVEHAKWFPRCEFLIRMKGQEFVDEIQGRYPHLLEQLLSTSDTTGEENADPPIIHFGPGESSSEDAVMMNTPVVKSALEMGFNRDLVKQTVQSKILTTGENYKTVNDIVSALLNAEDEKREEEKEKQAEEMASDDLSLIRKNRMALFQQLTCVLPILDNLLKANVINKQEHDIIKQKTQIPLQARELIDTILVKGNAAANIFKNCLKEIDSTLYKNLFVDKNMKYIPTEDVSGLSLEEQLRRLQEERTCKVCMDKEVSVVFIPCGHLVVCQECAPSLRKCPICRGIIKGTVRTFLS",
# BIRC3_HUMAN
"MNIVENSIFLSNLMKSANTFELKYDLSCELYRMSTYSTFPAGVPVSERSLARAGFYYTGVNDKVKCFCCGLMLDNWKRGDSPTEKHKKLYPSCRFVQSLNSVNNLEATSQPTFPSSVTNSTHSLLPGTENSGYFRGSYSNSPSNPVNSRANQDFSALMRSSYHCAMNNENARLLTFQTWPLTFLSPTDLAKAGFYYIGPGDRVACFACGGKLSNWEPKDNAMSEHLRHFPKCPFIENQLQDTSRYTVSNLSMQTHAARFKTFFNWPSSVLVNPEQLASAGFYYVGNSDDVKCFCCDGGLRCWESGDDPWVQHAKWFPRCEYLIRIKGQEFIRQVQASYPHLLEQLLSTSDSPGDENAESSIIHFEPGEDHSEDAIMMNTPVINAAVEMGFSRSLVKQTVQRKILATGENYRLVNDLVLDLLNAEDEIREEERERATEEKESNDLLLIRKNRMALFQHLTCVIPILDSLLTAGIINEQEHDVIKQKTQTSLQARELIDTILVKGNIAATVFRNSLQEAEAVLYEHLFVQQDIKYIPTEDVSDLPVEEQLRRLQEERTCKVCMDKEVSIVFIPCGHLVVCKDCAPSLRKCPICRSTIKGTVRTFLS",
# BIRC7_HUMAN
"MGPKDSAKCLHRGPQPSHWAAGDGPTQERCGPRSLGSPVLGLDTCRAWDHVDGQILGQLRPLTEEEEEEGAGATLSRGPAFPGMGSEELRLASFYDWPLTAEVPPELLAAAGFFHTGHQDKVRCFFCYGGLQSWKRGDDPWTEHAKWFPSCQFLLRSKGRDFVHSVQETHSQLLGSWDPWEEPEDAAPVAPSVPASGYPELPTPRREVQSESAQEPGGVSPAEAQRAWWVLEPPGARDVEAQLRRLQEERTCKVCLDRAVSIVFVPCGHLVCAECAPGLQLCPICRAPVRSRVRTFLS",
# BIRC8_HUMAN
"MTGYEARLITFGTWMYSVNKEQLARAGFYAIGQEDKVQCFHCGGGLANWKPKEDPWEQHAKWYPGCKYLLEEKGHEYINNIHLTRSLEGALVQTTKKTPSLTKRISDTIFPNPMLQEAIRMGFDFKDVKKIMEERIQTSGSNYKTLEVLVADLVSAQKDTTENELNQTSLQREISPEEPLRRLQEEKLCKICMDRHIAVVFIPCGHLVTCKQCAEAVDRCPMCSAVIDFKQRVFMS",
# BMI1_HUMAN
"MHRTTRIKITELNPHLMCVLCGGYFIDATTIIECLHSFCKTCIVRYLETSKYCPICDVQVHKTRPLLNIRSDKTLQDIVYKLVPGLFKNEMKRRRDFYAAHPSADAANGSNEDRGEVADEDKRIITDDEIISLSIEFFDQNRLDRKVNKDKEKSKEEVNDKRYLRCPAAMTVMHLRKFLRSKMDIPNTFQIDVMYEEEPLKDYYTLMDIAYIYTWRRNGPLPLKYRVRPTCKRMKISHQRDGLTNAGELESDSGSDKANSPAGGIPSTSSCLPSPSTPVQSPHPQFPHISSTMNGTSNSPSGNHQSSFANRPRKSSVNGSSATSSG",
# BRAP_HUMAN
"MSVSLVVIRLELAEHSPVPAGFGFSAAAGEMSDEEIKKTTLASAVACLEGKSPGEKVAIIHQHLGRREMTDVIIETMKSNPDELKTTVEERKSSEASPTAQRSKDHSKECINAAPDSPSKQLPDQISFFSGNPSVEIVHGIMHLYKTNKMTSLKEDVRRSAMLCILTVPAAMTSHDLMKFVAPFNEVIEQMKIIRDSTPNQYMVLIKFRAQADADSFYMTCNGRQFNSIEDDVCQLVYVERAEVLKSEDGASLPVMDLTELPKCTVCLERMDESVNGILTTLCNHSFHSQCLQRWDDTTCPVCRYCQTPEPVEENKCFECGVQENLWICLICGHIGCGRYVSRHAYKHFEETQHTYAMQLTNHRVWDYAGDNYVHRLVASKTDGKIVQYECEGDTCQEEKIDALQLEYSYLLTSQLESQRIYWENKIVRIEKDTAEEINNMKTKFKETIEKCDNLEHKLNDLLKEKQSVERKCTQLNTKVAKLTNELKEEQEMNKCLRANQVLLQNKLKEEERVLKETCDQKDLQITEIQEQLRDVMFYLETQQKINHLPAETRQEIQEGQINIAMASASSPASSGGSGKLPSRKGRSKRGK",
# BRCA1_HUMAN
"MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFCLSSQFRGNETGLITPNKHGLLQNPYRIPPLFPIKSFVKTKCKKNLLEENFEEHSMSPEREMGNENIPSTVSTISRNNIRENVFKEASSSNINEVGSSTNEVGSSINEIGSSDENIQAELGRNRGPKLNAMLRLGVLQPEVYKQSLPGSNCKHPEIKKQEYEEVVQTVNTDFSPYLISDNLEQPMGSSHASQVCSETPDDLLDDGEIKEDTSFAENDIKESSAVFSKSVQKGELSRSPSPFTHTHLAQGYRRGAKKLESSEENLSSEDEELPCFQHLLFGKVNNIPSQSTRHSTVATECLSKNTEENLLSLKNSLNDCSNQVILAKASQEHHLSEETKCSASLFSSQCSELEDLTANTNTQDPFLIGSSKQMRHQSESQGVGLSDKELVSDDEERGTGLEENNQEEQSMDSNLGEAASGCESETSVSEDCSGLSSQSDILTTQQRDTMQHNLIKLQQEMAELEAVLEQHGSQPSNSYPSIISDSSALEDLRNPEQSTSEKAVLTSQKSSEYPISQNPEGLSADKFEVSADSSTSKNKEPGVERSSPSKCPSLDDRWYMHSCSGSLQNRNYPSQEELIKVVDVEEQQLEESGPHDLTETSYLPRQDLEGTPYLESGISLFSDDPESDPSEDRAPESARVGNIPSSTSALKVPQLKVAESAQSPAAAHTTDTAGYNAMEESVSREKPELTASTERVNKRMSMVVSGLTPEEFMLVYKFARKHHITLTNLITEETTHVVMKTDAEFVCERTLKYFLGIAGGKWVVSYFWVTQSIKERKMLNEHDFEVRGDVVNGRNHQGPKRARESQDRKIFRGLEICCYGPFTNMPTDQLEWMVQLCGASVVKELSSFTLGTGVHPIVVVQPDAWTEDNGFHAIGQMCEAPVVTREWVLDSVALYQCQELDTYLIPQIPHSHY",
# CBL_HUMAN
"MAGNVKKSSGAGGGSGSGGSGSGGLIGLMKDAFQPHHHHHHHLSPHPPGTVDKKMVEKCWKLMDKVVRLCQNPKLALKNSPPYILDLLPDTYQHLRTILSRYEGKMETLGENEYFRVFMENLMKKTKQTISLFKEGKERMYEENSQPRRNLTKLSLIFSHMLAELKGIFPSGLFQGDTFRITKADAAEFWRKAFGEKTIVPWKSFRQALHEVHPISSGLEAMALKSTIDLTCNDYISVFEFDIFTRLFQPWSSLLRNWNSLAVTHPGYMAFLTYDEVKARLQKFIHKPGSYIFRLSCTRLGQWAIGYVTADGNILQTIPHNKPLFQALIDGFREGFYLFPDGRNQNPDLTGLCEPTPQDHIKVTQEQYELYCEMGSTFQLCKICAENDKDVKIEPCGHLMCTSCLTSWQESEGQGCPFCRCEIKGTEPIVVDPFDPRGSGSLLRQGAEGAPSPNYDDDDDERADDTLFMMKELAGAKVERPPSPFSMAPQASLPPVPPRLDLLPQRVCVPSSASALGTASKAASGSLHKDKPLPVPPTLRDLPPPPPPDRPYSVGAESRPQRRPLPCTPGDCPSRDKLPPVPSSRLGDSWLPRPIPKVPVSAPSSSDPWTGRELTNRHSLPFSLPSQMEPRPDVPRLGSTFSLDTSMSMNSSPLVGPECDHPKIKPSSSANAIYSLAARPLPVPKLPPGEQCEGEEDTEYMTPSSRPLRPLDTSQSSRACDCDQQIDSCTYEAMYNIQSQAPSITESSTFGEGNLAAAHANTGPEESENEDDGYDVPKPPVPAVLARRTLSDISNASSSFGWLSLDGDPTTNVTEGSQVPERPPKPFPRRINSERKAGSCQQGSGPAASAATASPQLSSEIENLMSQGYSYQDIQKALVIAQNNIEMAKNILREFVSISSPAHVAT",
# CBLB_HUMAN
"MANSMNGRNPGGRGGNPRKGRILGIIDAIQDAVGPPKQAAADRRTVEKTWKLMDKVVRLCQNPKLQLKNSPPYILDILPDTYQHLRLILSKYDDNQKLAQLSENEYFKIYIDSLMKKSKRAIRLFKEGKERMYEEQSQDRRNLTKLSLIFSHMLAEIKAIFPNGQFQGDNFRITKADAAEFWRKFFGDKTIVPWKVFRQCLHEVHQISSGLEAMALKSTIDLTCNDYISVFEFDIFTRLFQPWGSILRNWNFLAVTHPGYMAFLTYDEVKARLQKYSTKPGSYIFRLSCTRLGQWAIGYVTGDGNILQTIPHNKPLFQALIDGSREGFYLYPDGRSYNPDLTGLCEPTPHDHIKVTQEQYELYCEMGSTFQLCKICAENDKDVKIEPCGHLMCTSCLTAWQESDGQGCPFCRCEIKGTEPIIVDPFDPRDEGSRCCSIIDPFGMPMLDLDDDDDREESLMMNRLANVRKCTDRQNSPVTSPGSSPLAQRRKPQPDPLQIPHLSLPPVPPRLDLIQKGIVRSPCGSPTGSPKSSPCMVRKQDKPLPAPPPPLRDPPPPPPERPPPIPPDNRLSRHIHHVESVPSRDPPMPLEAWCPRDVFGTNQLVGCRLLGEGSPKPGITASSNVNGRHSRVGSDPVLMRKHRRHDLPLEGAKVFSNGHLGSEEYDVPPRLSPPPPVTTLLPSIKCTGPLANSLSEKTRDPVEEDDDEYKIPSSHPVSLNSQPSHCHNVKPPVRSCDNGHCMLNGTHGPSSEKKSNIPDLSIYLKGDVFDSASDPVPLPPARPPTRDNPKHGSSLNRTPSDYDLLIPPLGEDAFDALPPSLPPPPPPARHSLIEHSKPPGSSSRPSSGQDLFLLPSDPFVDLASGQVPLPPARRLPGENVKTNRTSQDYDQLPSCSDGSQAPARPPKPRPRRTAPEIHHRKPHGPEAALENVDAKIAKLMGEGYAFEEVKRALEIAQNNVEVARSILREFAFPPPVSPRLNL",
# CBLC_HUMAN
"MALAVAPWGRQWEEARALGRAVRMLQRLEEQCVDPRLSVSPPSLRDLLPRTAQLLREVAHSRRAAGGGGPGGPGGSGDFLLIYLANLEAKSRQVAALLPPRGRRSANDELFRAGSRLRRQLAKLAIIFSHMHAELHALFPGGKYCGHMYQLTKAPAHTFWRESCGARCVLPWAEFESLLGTCHPVEPGCTALALRTTIDLTCSGHVSIFEFDVFTRLFQPWPTLLKNWQLLAVNHPGYMAFLTYDEVQERLQACRDKPGSYIFRPSCTRLGQWAIGYVSSDGSILQTIPANKPLSQVLLEGQKDGFYLYPDGKTHNPDLTELGQAEPQQRIHVSEEQLQLYWAMDSTFELCKICAESNKDVKIEPCGHLLCSCCLAAWQHSDSQTCPFCRCEIKGWEAVSIYQFHGQATAEDSGNSSDQEGRELELGQVPLSAPPLPPRPDLPPRKPRNAQPKVRLLKGNSPPAALGPQDPAPA",
# HAKAI_HUMAN
"MDHTDNELQGTNSSGSLGGLDVRRRIPIKLISKQANKAKPAPRTQRTINRMPAKAPPGDEEGFDYNEEERYDCKGGELFANQRRFPGHLFWDFQINILGEKDDTPVHFCDKCGLPIKIYGRMIPCKHVFCYDCAILHEKKGDKMCPGCSDPVQRIEQCTRGSLFMCSIVQGCKRTYLSQRDLQAHINHRHMRAGKPVTRASLENVHPPIAPPPTEIPERFIMPPDKHHMSHIPPKQHIMMPPPPLQHVPHEHYNQPHEDIRAPPAELSMAPPPPRSVSQETFRISTRKHSNLITVPIQDDSNSGAREPPPPAPAPAHHHPEYQGQPVVSHPHHIMPPQQHYAPPPPPPPPISHPMPHPPQAAGTPHLVYSQAPPPPMTSAPPPITPPPGHIIAQMPPYMNHPPPGPPPPQHGGPPVTAPPPHHYNPNSLPQFTEDQGTLSPPFTQPGGMSPGIWPAPRGPPPPPRLQGPPSQTPLPGPHHPDQTRYRPYYQ",
# IHO1_HUMAN
"MNFNVWNIKEMLSIPSGSGNKKSSNWNNNQNDYSSLSDSQFLFGSQFCPENSETLSAPLDFGAHLRHSKQSQQNYLEGEPSIFTKYQTKPQLFGGDIKDGGLFPPPLSVGKSKGLLEQFEEKKKRAKDKCDSETLYNFVSNVRESILRLQTSVEKSEDHLSSRSQSILDSLETVAKTLQETIQAQNDLVFEAVQDKGNMEQAILEMKKRFEARQGEFIEMKSNLKHLEVLVAQQSQEFQQLCEQLGQLNVPSVLAELKRLISVPPVKDSASQTSPPLAQSLNLTRQEKYTSEKPVLWQAQALPAAWNPGMGSLQPGEFDVWGEGAKNDDLQEEAALPAFGSHERNRHVKDKVVQTNCKNWAVTKTGAKNHGSSVPGHKIPSDRDLVSQGASQLTSLEINFSTSIKNACQKYQAQSMFLCDPREHLVIKQKDGTVEMRGKDKKQQPRKAHRAHRGRLIASKQKQIPIQTCKFNSKYQSPQPAISVPQSPFLGQQEPRAQPLHLQCPRSPRKPVCPILGGTVMPNKTVRAVQGRLLQLSRCSSQDNWLLSSSSQGDHQMSWFSDLNLGCSETPLCKEAGKNLLYDLGFDSSDDDGF",
# CIP1_HUMAN
"MSLCEDMLLCNYRKCRIKLSGYAWVTACSHIFCDQHGSGEFSRSPAICPACNSTLSGKLDIVRTELSPSEEYKAMVLAGLRPEIVLDISSRALAFWTYQVHQERLYQEYNFSKAEGHLKQMEKIYTQQIQSKDVELTSMKGEVTSMKKVLEEYKKKFSDISEKLMERNRQYQKLQGLYDSLRLRNITIANHEGTLEPSMIAQSGVLGFPLGNNSKFPLDNTPVRNRGDGDGDFQFRPFFAGSPTAPEPSNSFFSFVSPSRELEQQQVSSRAFKVKRI",
# CGRF1_HUMAN
"MAAVFLVTLYEYSPLFYIAVVFTCFIVTTGLVLGWFGWDVPVILRNSEETQFSTRVFKKQMRQVKNPFGLEITNPSSASITTGITLTTDCLEDSLLTCYWGCSVQKLYEALQKHVYCFRISTPQALEDALYSEYLYQEQYFIKKDSKEEIYCQLPRDTKIEDFGTVPRSRYPLVALLTLADEDDREIYDIISMVSVIHIPDRTYKLSCRILYQYLLLAQGQFHDLKQLFMSANNNFTPSNNSSSEEKNTDRSLLEKVGLSESEVEPSEENSKDCVVCQNGTVNWVLLPCRHTCLCDGCVKYFQQCPMCRQFVQESFALCSQKEQDKDKPKTL",
# CHFR_HUMAN
"MERPEEGKQSPPPQPWGRLLRLGAEEGEPHVLLRKREWTIGRRRGCDLSFPSNKLVSGDHCRIVVDEKSGQVTLEDTSTSGTVINKLKVVKKQTCPLQTGDVIYLVYRKNEPEHNVAYLYESLSEKQGMTQESFEANKENVFHGTKDTSGAGAGRGADPRVPPSSPATQVCFEEPQPSTSTSDLFPTASASSTEPSPAGRERSSSCGSGGGGISPKGSGPSVASDEVSSFASALPDRKTASFSSLEPQDQEDLEPVKKKMRGDGDLDLNGQLLVAQPRRNAQTVHEDVRAAAGKPDKMEETLTCIICQDLLHDCVSLQPCMHTFCAACYSGWMERSSLCPTCRCPVERICKNHILNNLVEAYLIQHPDKSRSEEDVQSMDARNKITQDMLQPKVRRSFSDEEGSSEDLLELSDVDSESSDISQPYVVCRQCPEYRRQAAQPPHCPAPEGEPGAPQALGDAPSTSVSLTTAVQDYVCPLQGSHALCTCCFQPMPDRRAEREQDPRVAPQQCAVCLQPFCHLYWGCTRTGCYGCLAPFCELNLGDKCLDGVLNNNSYESDILKNYLATRGLTWKNMLTESLVALQRGVFLLSDYRVTGDTVLCYCCGLRSFRELTYQYRQNIPASELPVAVTSRPDCYWGRNCRTQVKAHHAMKFNHICEQTRFKN",
# CNOT4_HUMAN
"MSRSPDAKEDPVECPLCMEPLEIDDINFFPCTCGYQICRFCWHRIRTDENGLCPACRKPYPEDPAVYKPLSQEELQRIKNEKKQKQNERKQKISENRKHLASVRVVQKNLVFVVGLSQRLADPEVLKRPEYFGKFGKIHKVVINNSTSYAGSQGPSASAYVTYIRSEDALRAIQCVNNVVVDGRTLKASLGTTKYCSYFLKNMQCPKPDCMYLHELGDEAASFTKEEMQAGKHQEYEQKLLQELYKLNPNFLQLSTGSVDKNKNKVTPLQRYDTPIDKPSDSLSIGNGDNSQQISNSDTPSPPPGLSKSNPVIPISSSNHSARSPFEGAVTESQSLFSDNFRHPNPIPSGLPPFPSSPQTSSDWPTAPEPQSLFTSETIPVSSSTDWQAAFGFGSSKQPEDDLGFDPFDVTRKALADLIEKELSVQDQPSLSPTSLQNSSSHTTTAKGPGSGFLHPAAATNANSLNSTFSVLPQRFPQFQQHRAVYNSFSFPGQAARYPWMAFPRNSIMHLNHTANPTSNSNFLDLNLPPQHNTGLGGIPVAGEEEVKVSTMPLSTSSHSLQQGQQPTSLHTTVA",
# CUL9_HUMAN
"MVGERHAGDLMVPLGPRLQAYPEELIRQRPGHDGHPEYLIRWSVLKCGEVGKVGVEEGKAEHILMWLSAPEVYANCPGLLGERALSKGLQHEPAGVSGSFPRDPGGLDEVAMGEMEADVQALVRRAARQLAESGTPSLTAAVLHTIHVLSAYASIGPLTGVFRETGALDLLMHMLCNPEPQIRRSAGKMLQALAAHDAGSRAHVLLSLSQQDGIEQHMDFDSRYTLLELFAETTSSEEHCMAFEGIHLPQIPGKLLFSLVKRYLCVTSLLDQLNSSPELGAGDQSSPCATREKSRGQRELEFSMAVGNLISELVRSMGWARNLSEQGMSPPRPTRSIFQPYISGPSLLLPTIVTTPRRQGWVFRQRSEFSSRSGYGEYVQQTLQPGMRVRMLDDYEEISAGDEGEFRQSNNGIPPVQVFWQSTGRTYWVHWHMLEILGPEEATEDKASAAVEKGAGATVLGTAFPSWDWNPMDGLYPLPYLQPEPQKNERVGYLTQAEWWELLFFIKKLDLCEQQPIFQNLWKNLDETLGEKALGEISVSVEMAESLLQVLSSRFEGSTLNDLLNSQIYTKYGLLSNEPSSSSTSRNHSCTPDPEEESKSEASFSEEETESLKAKAEAPKTEAEPTKTRTETPMAQSDSQLFNQLLVTEGMTLPTEMKEAASEMARALRGPGPRSSLDQHVAAVVATVQISSLDTNLQLSGLSALSQAVEEVTERDHPLVRPDRSLREKLVKMLVELLTNQVGEKMVVVQALRLLYLLMTKHEWRPLFAREGGIYAVLVCMQEYKTSVLVQQAGLAALKMLAVASSSEIPTFVTGRDSIHSLFDAQMTREIFASIDSATRPGSESLLLTVPAAVILMLNTEGCSSAARNGLLLLNLLLCNHHTLGDQIITQELRDTLFRHSGIAPRTEPMPTTRTILMMLLNRYSEPPGSPERAALETPIIQGQDGSPELLIRSLVGGPSAELLLDLERVLCREGSPGGAVRPLLKRLQQETQPFLLLLRTLDAPGPNKTLLLSVLRVITRLLDFPEAMVLPWHEVLEPCLNCLSGPSSDSEIVQELTCFLHRLASMHKDYAVVLCCLGAKEILSKVLDKHSAQLLLGCELRDLVTECEKYAQLYSNLTSSILAGCIQMVLGQIEDHRRTHQPINIPFFDVFLRHLCQGSSVEVKEDKCWEKVEVSSNPHRASKLTDHNPKTYWESNGSTGSHYITLHMHRGVLVRQLTLLVASEDSSYMPARVVVFGGDSTSCIGTELNTVNVMPSASRVILLENLNRFWPIIQIRIKRCQQGGIDTRVRGVEVLGPKPTFWPLFREQLCRRTCLFYTIRAQAWSRDIAEDHRRLLQLCPRLNRVLRHEQNFADRFLPDDEAAQALGKTCWEALVSPLVQNITSPDAEGVSALGWLLDQYLEQRETSRNPLSRAASFASRVRRLCHLLVHVEPPPGPSPEPSTRPFSKNSKGRDRSPAPSPVLPSSSLRNITQCWLSVVQEQVSRFLAAAWRAPDFVPRYCKLYEHLQRAGSELFGPRAAFMLALRSGFSGALLQQSFLTAAHMSEQFARYIDQQIQGGLIGGAPGVEMLGQLQRHLEPIMVLSGLELATTFEHFYQHYMADRLLSFGSSWLEGAVLEQIGLCFPNRLPQLMLQSLSTSEELQRQFHLFQLQRLDKLFLEQEDEEEKRLEEEEEEEEEEEAEKELFIEDPSPAISILVLSPRCWPVSPLCYLYHPRKCLPTEFCDALDRFSSFYSQSQNHPVLDMGPHRRLQWTWLGRAELQFGKQILHVSTVQMWLLLKFNQTEEVSVETLLKDSDLSPELLLQALVPLTSGNGPLTLHEGQDFPHGGVLRLHEPGPQRSGEALWLIPPQAYLNVEKDEGRTLEQKRNLLSCLLVRILKAHGEKGLHIDQLVCLVLEAWQKGPNPPGTLGHTVAGGVACTSTDVLSCILHLLGQGYVKRRDDRPQILMYAAPEPMGPCRGQADVPFCGSQSETSKPSPEAVATLASLQLPAGRTMSPQEVEGLMKQTVRQVQETLNLEPDVAQHLLAHSHWGAEQLLQSYSEDPEPLLLAAGLCVHQAQAVPVRPDHCPVCVSPLGCDDDLPSLCCMHYCCKSCWNEYLTTRIEQNLVLNCTCPIADCPAQPTGAFIRAIVSSPEVISKYEKALLRGYVESCSNLTWCTNPQGCDRILCRQGLGCGTTCSKCGWASCFNCSFPEAHYPASCGHMSQWVDDGGYYDGMSVEAQSKHLAKLISKRCPSCQAPIEKNEGCLHMTCAKCNHGFCWRCLKSWKPNHKDYYNCSAMVSKAARQEKRFQDYNERCTFHHQAREFAVNLRNRVSAIHEVPPPRSFTFLNDACQGLEQARKVLAYACVYSFYSQDAEYMDVVEQQTENLELHTNALQILLEETLLRCRDLASSLRLLRADCLSTGMELLRRIQERLLAILQHSAQDFRVGLQSPSVEAWEAKGPNMPGSQPQASSGPEAEEEEEDDEDDVPEWQQDEFDEELDNDSFSYDESENLDQETFFFGDEEEDEDEAYD",
# TM276_HUMAN
"MAPKPGAEWSTALSHLVLGVVSLHAAVSTAEASRGAAAGFLLQVLAATTTLAPGLSTHEDCLAGAWVATVIGLPLLAFDFHWVNGDRSSANLLLGGGMVLAVAGGHLGPEGRSVAGQAMLLVVAVTILIVAVFTANTYGMWGGAMLGVAGLLSRLEEDRLLLLPKEDVCRWALAVGSWAYCRALHTQRLQWE",
# ZTRF1_HUMAN
"MSGAEEAGGGGPAAGPAGSVPAGVGVGVGAGPGAAAGQAAAAALGEAAGPGLPDEAGLAGARQLQLQEAAGDPDAPPKKRLRAAEAAEAAAAAAAAGSGKLEERLYSVLCCTVCLDLPKASVYQCTNGHLMCAGCFIHLLADARLKEEQATCPNCRCEISKSLCCRNLAVEKAVSELPSECGFCLRQFPRSLLERHQKEECQDRVTQCKYKRIGCPWHGPFHELTVHEAACAHPTKTGSELMEILDGMDQSHRKEMQLYNSIFSLLSFEKIGYTEVQFRPYRTDDFITRLYYETPRFTVLNQTWVLKARVNDSERNPNLSCKRTLSFQLLLKSKVTAPLECSFLLLKGPYDDVRISPVIYHFVFTNESNETDYVPLPIIDSVECNKLLAAKNINLRLFLFQIQK",
# DCST1_HUMAN
"MDIKHHQNGTRGQRRKQPHTTVQRLLTWGLPVSCSWFLWRQPGEFPVTALLLGAGAGGLLAIGLFQLLVNPMNIYEEQKIMFLYSLVGLGAMGWGTSPHIRCASLLLVPKMLGKEGRLFVLGYALAAIYVGPVANLRHNLNNVIASLGCTVELQINNTRAAWRISTAPLRAMFKDLLSSKELLRAETRNISATFEDLDAQVNSETGYTPEDTMDSGETAQGREARQAPASRLHLSTQKMYELKTKLRCSYVVNQAILSCRRWFDRKHEQCMKHIWVPLLTHLLCLPMKFKFFCGIAKVMEVWCRNRIPVEGNFGQTYDSLNQSIRGLDGEFSANIDFKEEKQAGVLGLNTSWERVSTEVRDYVYRQEARLEWALGLLHVLLSCTFLLVLHASFSYMDSYNHDIRFDNIYISTYFCQIDDRRKKLGKRTLLPLRKAEEKTVIFPCKPTIQASEMSNVVRELLETLPILLLLVVLCGLDWALYSIFDTIRHHSFLQYSFRSSHKLEVKVGGDSMLARLLRKTIGALNTSSETVMESNNMPCLPQPVGLDARAYWRAAVPIGLLVCLCLLQAFGYRLRRVIAAFYFPKREKKRILFLYNDLLKKRAAFTKLRRAAILRRERQQKAPRHPLADILHRGCPLLRRWLCRRCVVCQAPETPESYVCRTLDCEAVYCWSCWDDMRQRCPVCTPREELSSSAFSDSNDDTAYAG",
# DTX1_HUMAN
"MSRPGHGGLMPVNGLGFPPQNVARVVVWEWLNEHSRWRPYTATVCHHIENVLKEDARGSVVLGQVDAQLVPYIIDLQSMHQFRQDTGTMRPVRRNFYDPSSAPGKGIVWEWENDGGAWTAYDMDICITIQNAYEKQHPWLDLSSLGFCYLIYFNSMSQMNRQTRRRRRLRRRLDLAYPLTVGSIPKSQSWPVGASSGQPCSCQQCLLVNSTRAASNAILASQRRKAPPAPPLPPPPPPGGPPGALAVRPSATFTGAALWAAPAAGPAEPAPPPGAPPRSPGAPGGARTPGQNNLNRPGPQRTTSVSARASIPPGVPALPVKNLNGTGPVHPALAGMTGILLCAAGLPVCLTRAPKPILHPPPVSKSDVKPVPGVPGVCRKTKKKHLKKSKNPEDVVRRYMQKVKNPPDEDCTICMERLVTASGYEGVLRHKGVRPELVGRLGRCGHMYHLLCLVAMYSNGNKDGSLQCPTCKAIYGEKTGTQPPGKMEFHLIPHSLPGFPDTQTIRIVYDIPTGIQGPEHPNPGKKFTARGFPRHCYLPNNEKGRKVLRLLITAWERRLIFTIGTSNTTGESDTVVWNEIHHKTEFGSNLTGHGYPDASYLDNVLAELTAQGVSEAAAKA",
# DTX2_HUMAN
"MAMAPSPSLVQVYTSPAAVAVWEWQDGLGTWHPYSATVCSFIEQQFVQQKGQRFGLGSLAHSIPLGQADPSLAPYIIDLPSWTQFRQDTGTMRAVRRHLFPQHSAPGRGVVWEWLSDDGSWTAYEASVCDYLEQQVARGNQLVDLAPLGYNYTVNYTTHTQTNKTSSFCRSVRRQAGPPYPVTTIIAPPGHTGVACSCHQCLSGSRTGPVSGRYRHSMTNLPAYPVPQHPPHRTASVFGTHQAFAPYNKPSLSGARSAPRLNTTNAWGAAPPSLGSQPLYRSSLSHLGPQHLPPGSSTSGAVSASLPSGPSSSPGSVPATVPMQMPKPSRVQQALAGMTSVLMSAIGLPVCLSRAPQPTSPPASRLASKSHGSVKRLRKMSVKGATPKPEPEPEQVIKNYTEELKVPPDEDCIICMEKLSTASGYSDVTDSKAIGSLAVGHLTKCSHAFHLLCLLAMYCNGNKDGSLQCPSCKTIYGEKTGTQPQGKMEVLRFQMSLPGHEDCGTILIVYSIPHGIQGPEHPNPGKPFTARGFPRQCYLPDNAQGRKVLELLKVAWKRRLIFTVGTSSTTGETDTVVWNEIHHKTEMDRNITGHGYPDPNYLQNVLAELAAQGVTEDCLEQQ",
# DTX3_HUMAN
"MSFVLSRMAACGGTCKNKVTVSKPVWDFLSKETPARLARLREEHRVSILIDGETSDIYVLQLSPQGPPPAPPNGLYLARKALKGLLKEAEKELKKAQRQGELMGCLALGGGGEHPEMHRAGPPPLRAAPLLPPGARGLPPPPPPLPPPLPPRLREEAEEQESTCPICLGEIQNAKTLEKCRHSFCEGCITRALQVKKACPMCGRFYGQLVGNQPQNGRMLVSKDATLLLPSYEKYGTIVIQYVFPPGVQGAEHPNPGVRYPGTTRVAYLPDCPEGNKVLTLFRKAFDQRLTFTIGTSMTTGRPNVITWNDIHHKTSCTGGPQLFGYPDPTYLTRVQEELRAKGITDD",
# DTX3L_HUMAN
"MASHLRPPSPLLVRVYKSGPRVRRKLESYFQSSKSSGGGECTVSTQEHEAPGTFRVEFSERAAKERVLKKGEHQILVDEKPVPIFLVPTENSIKKNTRPQISSLTQSQAETPSGDMHQHEGHIPNAVDSCLQKIFLTVTADLNCNLFSKEQRAYITTLCPSIRKMEGHDGIEKVCGDFQDIERIHQFLSEQFLESEQKQQFSPSMTERKPLSQQERDSCISPSEPETKAEQKSNYFEVPLPYFEYFKYICPDKINSIEKRFGVNIEIQESSPNMVCLDFTSSRSGDLEAARESFASEFQKNTEPLKQECVSLADSKQANKFKQELNHQFTKLLIKEKGGELTLLGTQDDISAAKQKISEAFVKIPVKLFAANYMMNVIEVDSAHYKLLETELLQEISEIEKRYDICSKVSEKGQKTCILFESKDRQVDLSVHAYASFIDAFQHASCQLMREVLLLKSLGKERKHLHQTKFADDFRKRHPNVHFVLNQESMTLTGLPNHLAKAKQYVLKGGGMSSLAGKKLKEGHETPMDIDSDDSKAASPPLKGSVSSEASELDKKEKGICVICMDTISNKKVLPKCKHEFCAPCINKAMSYKPICPTCQTSYGIQKGNQPEGSMVFTVSRDSLPGYESFGTIVITYSMKAGIQTEEHPNPGKRYPGIQRTAYLPDNKEGRKVLKLLYRAFDQKLIFTVGYSRVLGVSDVITWNDIHHKTSRFGGPEMYGYPDPSYLKRVKEELKAKGIE",
# DTX4_HUMAN
"MLLASAVVVWEWLNEHGRWRPYSPAVSHHIEAVVRAGPRAGGSVVLGQVDSRLAPYIIDLQSMNQFRQDTGTLRPVRRNYYDPSSAPGKGVVWEWENDNGSWTPYDMEVGITIQHAYEKQHPWIDLTSIGFSYVIDFNTMGQINRQTQRQRRVRRRLDLIYPMVTGTLPKAQSWPVSPGPATSPPMSPCSCPQCVLVMSVKAAVVNGSTGPLQLPVTRKNMPPPGVVKLPPLPGSGAKPLDSTGTIRGPLKTAPSQVIRRQASSMPTGTTMGSPASPPGPNSKTGRVALATLNRTNLQRLAIAQSRVLIASGVPTVPVKNLNGSSPVNPALAGITGILMSAAGLPVCLTRPPKLVLHPPPVSKSEIKSIPGVSNTSRKTTKKQAKKGKTPEEVLKKYLQKVRHPPDEDCTICMERLTAPSGYKGPQPTVKPDLVGKLSRCGHVYHIYCLVAMYNNGNKDGSLQCPTCKTIYGVKTGTQPPGKMEYHLIPHSLPGHPDCKTIRIIYSIPPGIQGPEHPNPGKSFSARGFPRHCYLPDSEKGRKVLKLLLVAWDRRLIFAIGTSSTTGESDTVIWNEVHHKTEFGSNLTGHGYPDANYLDNVLAELAAQGISEDSTAQEKD",
# DZIP3_HUMAN
"MDSLPDEFFVRHPAVEDQRKEETENKLEKSSGQLNKQENDIPTDLVPVNLLLEVKKLLNAINTLPKGVVPHIKKFLQEDFSFQTMQREVAANSQNGEEIVPALTLRFLITQLEAALRNIQAGNYTAHQINIGYYLTLLFLYGVALTERGKKEDYTEAENKFLVMKMMIQENEICENFMSLVYFGRGLLRCAQKRYNGGLLEFHKSLQEIGDKNDHWFDIDPTEDEDLPTTFKDLLNNFIKTTESNIMKQTICSYLDCERSCEADILKNTSYKGFFQLMCSKSCCVYFHKICWKKFKNLKYPGENDQSFSGKKCLKEGCTGDMVRMLQCDVPGIVKILFEVVRKDEYITIENLGASYRKLISLKITDTDIRPKISLKFNTKDEMPIFKLDYNYFYHLLHIIIISGTDIVRQIFDEAMPPPLLKKELLIHKNVLESYYNHLWTNHPLGGSWHLLYPPNKELPQSKQFDLCLLLALIKHLNVFPAPKKGWNMEPPSSDISKSADILRLCKYRDILLSEILMNGLTESQFNSIWKKVSDILLRLGMMQEDIDKVKENPIENISLDYHQLSVYLGIPVPEIIQRMLSCYQQGIALQSITGSQRIEIEELQNEEEELSPPLMEYNINVKSHPEIQFAEINKDGTSIPSESSTESLKDLQEVKSKQRKKKKTKNKKNKDSKEDQVPYVVEKEEQLRKEQANPHSVSRLIKDDASDVQEDSAMEDKFYSLDELHILDMIEQGSAGKVTTDYGETEKERLARQRQLYKLHYQCEDFKRQLRTVTFRWQENQMQIKKKDKIIASLNQQVAFGINKVSKLQRQIHAKDNEIKNLKEQLSMKRSQWEMEKHNLESTMKTYVSKLNAETSRALTAEVYFLQCRRDFGLLHLEQTEKECLNQLARVTHMAASNLESLQLKAAVDSWNAIVADVRNKIAFLRTQYNEQINKVKQGFALSTLPPVQLPPPPPSPEILMQQFLGRPLVKESFFRPILTVPQMPAVCPGVVSATGQPRAPLMTGIAWALPAPVGDAVPPSAGLRSDPSIMNWERITDRLKTAFPQQTRKELTDFLRKLKDAYGKSLSELTFDEIVCKISQFIDPKKSQSQGKSVSNVNCVSPSHSPSQPDAAQPPKPAWRPLTSQGPATWEGASNPDEEEEEEEPCVICHENLSPENLSVLPCAHKFHAQCIRPWLMQQGTCPTCRLHVLLPEEFPGHPSRQLPKI",
# E4F1_HUMAN
"MEGAMAVRVTAAHTAEAQAEAGREAGEGAVAAVAAALAPSGFLGLPAPFSEEDEDDVHRCGRCQAEFTALEDFVQHKIQKACQRAPPEALPATPATTALLGQEVVPAAPGPEEPITVAHIVVEAASLAADISHASDLVGGGHIKEVIVAAEAELGDGEMAEAPGSPRQQGLGLAGEGEQAQVKLLVNKDGRYVCALCHKTFKTGSILKAHMVTHSSRKDHECKLCGASFRTKGSLIRHHRRHTDERPYKCSKCGKSFRESGALTRHLKSLTPCTEKIRFSVSKDVVVSKEDARAGSGAGAAGLGTATSSVTGEPIETSPVIHLVTDAKGTVIHEVHVQMQELSLGMKALAPEPPVSQELPCSSEGSRENLLHQAMQNSGIVLERAAGEEGALEPAPAAGSSPQPLAVAAPQLPVLEVQPLETQVASEASAVPRTHPCPQCSETFPTAATLEAHKRGHTGPRPFACAQCGKAFPKAYLLKKHQEVHVRERRFRCGDCGKLYKTIAHVRGHRRVHSDERPYPCPKCGKRYKTKNAQQVHFRTHLEEKPHVCQFCSRGFREKGSLVRHVRHHTGEKPFKCYKCGRGFAEHGTLNRHLRTKGGCLLEVEELLVSEDSPAAATTVLTEDPHTVLVEFSSVVADTQEYIIEATADDAETSEATEIIEGTQTEVDSHIMKVVQQIVHQASAGHQIIVQNVTMDEETALGPEAAAADTITIATPESLTEQVAMTLASAISEGTVLAARAGTSGTEQATVTMVSSEDIEILEHAGELVIASPEGQLEVQTVIV",
# FANCL_HUMAN
"MAVTEASLLRQCPLLLPQNRSKTVYEGFISAQGRDFHLRIVLPEDLQLKNARLLCSWQLRTILSGYHRIVQQRMQHSPDLMSFMMELKMLLEVALKNRQELYALPPPPQFYSSLIEEIGTLGWDKLVYADTCFSTIKLKAEDASGREHLITLKLKAKYPAESPDYFVDFPVPFCASWTPQSSLISIYSQFLAAIESLKAFWDVMDEIDEKTWVLEPEKPPRSATARRIALGNNVSINIEVDPRHPTMLPECFFLGADHVVKPLGIKLSRNIHLWDPENSVLQNLKDVLEIDFPARAILEKSDFTMDCGICYAYQLDGTIPDQVCDNSQCGQPFHQICLYEWLRGLLTSRQSFNIIFGECPYCSKPITLKMSGRKH",
# G2E3_HUMAN
"MNESKPGDSQNLACVFCRKHDDCPNKYGEKKTKEKWNLTVHYYCLLMSSGIWQRGKEEEGVYGFLIEDIRKEVNRASKLKCCVCKKNGASIGCVAPRCKRSYHFPCGLQRECIFQFTGNFASFCWDHRPVQIITSNNYRESLPCTICLEFIEPIPSYNILRSPCCKNAWFHRDCLQVQAINAGVFFFRCTICNNSDIFQKEMLRMGIHIPEKDASWELEENAYQELLQHYERCDVRRCRCKEGRDYNAPDSKWEIKRCQCCGSSGTHLACSSLRSWEQNWECLECRGIIYNSGEFQKAKKHVLPNSNNVGITDCLLEESSPKLPRQSPGSQSKDLLRQGSKFRRNVSTLLIELGFQIKKKTKRLYINKANIWNSALDAFRNRNFNPSYAIEVAYVIENDNFGSEHPGSKQEFLSLLMQHLENSSLFEGSLSKNLSLNSQALKENLYYEAGKMLAISLVHGGPSPGFFSKTLFNCLVYGPENTQPILDDVSDFDVAQIIIRINTATTVADLKSIINECYNYLELIGCLRLITTLSDKYMLVKDILGYHVIQRVHTPFESFKQGLKTLGVLEKIQAYPEAFCSILCHKPESLSAKILSELFTVHTLPDVKALGFWNSYLQAVEDGKSTTTMEDILIFATGCSSIPPAGFKPTPSIECLHVDFPVGNKCNNCLAIPITNTYKEFQENMDFTIRNTLRLEKEESSHYIGH",
# HACE1_HUMAN
"MERAMEQLNRLTRSLRRARTVELPEDNETAVYTLMPMVMADQHRSVSELLSNSKFDVNYAFGRVKRSLLHIAANCGSVECLVLLLKKGANPNYQDISGCTPLHLAARNGQKKCMSKLLEYSADVNICNNEGLTAIHWLAVNGRTELLHDLVQHVSDVDVEDAMGQTALHVACQNGHKTTVQCLLDSGADINRPNVSGATPLYFACSHGQRDTAQILLLRGAKYLPDKNGVTPLDLCVQGGYGETCEVLIQYHPRLFQTIIQMTQNEDLRENMLRQVLEHLSQQSESQYLKILTSLAEVATTNGHKLLSLSSNYDAQMKSLLRIVRMFCHVFRIGPSSPSNGIDMGYNGNKTPRSQVFKPLELLWHSLDEWLVLIATELMKNKRDSTEITSILLKQKGQDQDAASIPPFEPPGPGSYENLSTGTRESKPDALAGRQEASADCQDVISMTANRLSAVIQAFYMCCSCQMPPGMTSPRFIEFVCKHDEVLKCFVNRNPKIIFDHFHFLLECPELMSRFMHIIKAQPFKDRCEWFYEHLHSGQPDSDMVHRPVNENDILLVHRDSIFRSSCEVVSKANCAKLKQGIAVRFHGEEGMGQGVVREWFDILSNEIVNPDYALFTQSADGTTFQPNSNSYVNPDHLNYFRFAGQILGLALNHRQLVNIYFTRSFYKHILGIPVNYQDVASIDPEYAKNLQWILDNDISDLGLELTFSVETDVFGAMEEVPLKPGGGSILVTQNNKAEYVQLVTELRMTRAIQPQINAFLQGFHMFIPPSLIQLFDEYELELLLSGMPEIDVSDWIKNTEYTSGYEREDPVIQWFWEVVEDITQEERVLLLQFVTGSSRVPHGGFANIMGGSGLQNFTIAAVPYTPNLLPTSSTCINMLKLPEYPSKEILKDRLLVALHCGSYGYTMA",
# HECD1_HUMAN
"MADVDPDTLLEWLQMGQGDERDMQLIALEQLCMLLLMSDNVDRCFETCPPRTFLPALCKIFLDESAPDNVLEVTARAITYYLDVSAECTRRIVGVDGAIKALCNRLVVVELNNRTSRDLAEQCVKVLELICTRESGAVFEAGGLNCVLTFIRDSGHLVHKDTLHSAMAVVSRLCGKMEPQDSSLEICVESLSSLLKHEDHQVSDGALRCFASLADRFTRRGVDPAPLAKHGLTEELLSRMAAAGGTVSGPSSACKPGRSTTGAPSTTADSKLSNQVSTIVSLLSTLCRGSPVVTHDLLRSELPDSIESALQGDERCVLDTMRLVDLLLVLLFEGRKALPKSSAGSTGRIPGLRRLDSSGERSHRQLIDCIRSKDTDALIDAIDTGAFEVNFMDDVGQTLLNWASAFGTQEMVEFLCERGADVNRGQRSSSLHYAACFGRPQVAKTLLRHGANPDLRDEDGKTPLDKARERGHSEVVAILQSPGDWMCPVNKGDDKKKKDTNKDEEECNEPKGDPEMAPIYLKRLLPVFAQTFQQTMLPSIRKASLALIRKMIHFCSEALLKEVCDSDVGHNLPTILVEITATVLDQEDDDDGHLLALQIIRDLVDKGGDIFLDQLARLGVISKVSTLAGPSSDDENEEESKPEKEDEPQEDAKELQQGKPYHWRDWSIIRGRDCLYIWSDAAALELSNGSNGWFRFILDGKLATMYSSGSPEGGSDSSESRSEFLEKLQRARGQVKPSTSSQPILSAPGPTKLTVGNWSLTCLKEGEIAIHNSDGQQATILKEDLPGFVFESNRGTKHSFTAETSLGSEFVTGWTGKRGRKLKSKLEKTKQKVRTMARDLYDDHFKAVESMPRGVVVTLRNIATQLESSWELHTNRQCIESENTWRDLMKTALENLIVLLKDENTISPYEMCSSGLVQALLTVLNNSMDLDMKQDCSQLVERINVFKTAFSENEDDESRPAVALIRKLIAVLESIERLPLHLYDTPGSTYNLQILTRRLRFRLERAPGETALIDRTGRMLKMEPLATVESLEQYLLKMVAKQWYDFDRSSFVFVRKLREGQNFIFRHQHDFDENGIIYWIGTNAKTAYEWVNPAAYGLVVVTSSEGRNLPYGRLEDILSRDNSALNCHSNDDKNAWFAIDLGLWVIPSAYTLRHARGYGRSALRNWVFQVSKDGQNWTSLYTHVDDCSLNEPGSTATWPLDPPKDEKQGWRHVRIKQMGKNASGQTHYLSLSGFELYGTVNGVCEDQLGKAAKEAEANLRRQRRLVRSQVLKYMVPGARVIRGLDWKWRDQDGSPQGEGTVTGELHNGWIDVTWDAGGSNSYRMGAEGKFDLKLAPGYDPDTVASPKPVSSTVSGTTQSWSSLVKNNCPDKTSAAAGSSSRKGSSSSVCSVASSSDISLGSTKTERRSEIVMEHSIVSGADVHEPIVVLSSAENVPQTEVGSSSSASTSTLTAETGSENAERKLGPDSSVRTPGESSAISMGIVSVSSPDVSSVSELTNKEAASQRPLSSSASNRLSVSSLLAAGAPMSSSASVPNLSSRETSSLESFVRRVANIARTNATNNMNLSRSSSDNNTNTLGRNVMSTATSPLMGAQSFPNLTTPGTTSTVTMSTSSVTSSSNVATATTVLSVGQSLSNTLTTSLTSTSSESDTGQEAEYSLYDFLDSCRASTLLAELDDDEDLPEPDEEDDENEDDNQEDQEYEEVMILRRPSLQRRAGSRSDVTHHAVTSQLPQVPAGAGSRPIGEQEEEEYETKGGRRRTWDDDYVLKRQFSALVPAFDPRPGRTNVQQTTDLEIPPPGTPHSELLEEVECTPSPRLALTLKVTGLGTTREVELPLTNFRSTIFYYVQKLLQLSCNGNVKSDKLRRIWEPTYTIMYREMKDSDKEKENGKMGCWSIEHVEQYLGTDELPKNDLITYLQKNADAAFLRHWKLTGTNKSIRKNRNCSQLIAAYKDFCEHGTKSGLNQGAISTLQSSDILNLTKEQPQAKAGNGQNSCGVEDVLQLLRILYIVASDPYSRISQEDGDEQPQFTFPPDEFTSKKITTKILQQIEEPLALASGALPDWCEQLTSKCPFLIPFETRQLYFTCTAFGASRAIVWLQNRREATVERTRTTSSVRRDDPGEFRVGRLKHERVKVPRGESLMEWAENVMQIHADRKSVLEVEFLGEEGTGLGPTLEFYALVAAEFQRTDLGAWLCDDNFPDDESRHVDLGGGLKPPGYYVQRSCGLFTAPFPQDSDELERITKLFHFLGIFLAKCIQDNRLVDLPISKPFFKLMCMGDIKSNMSKLIYESRGDRDLHCTESQSEASTEEGHDSLSVGSFEEDSKSEFILDPPKPKPPAWFNGILTWEDFELVNPHRARFLKEIKDLAIKRRQILSNKGLSEDEKNTKLQELVLKNPSGSGPPLSIEDLGLNFQFCPSSRIYGFTAVDLKPSGEDEMITMDNAEEYVDLMFDFCMHTGIQKQMEAFRDGFNKVFPMEKLSSFSHEEVQMILCGNQSPSWAAEDIINYTEPKLGYTRDSPGFLRFVRVLCGMSSDERKAFLQFTTGCSTLPPGGLANLHPRLTVVRKVDATDASYPSVNTCVHYLKLPEYSSEEIMRERLLAATMEKGFHLN",
# HECD2_HUMAN
"MSEAVRVPSPATPLVVAAPAPEERKGKESEREKLPPIVSAGAGATAGLDRGAKGQISTFSSFISAVSPKKEAAENRSSPAHLVFPNIKNVREPPPICLDVRQKQRTSMDASSSEMKAPVLPEPILPIQPKTVKDFQEDVEKVKSSGDWKAVHDFYLTTFDSFPELNAAFKKDATASFNTIEDSGINAKFVNAVYDTLLNTPQDVQKTVLKGIINSLLREWKGPRTKDDLRAYFILLQNPQFNNTSTYVIYAHLLRQIATLVEADHHFLVHWFKKLSQKRFKQLVERLLQFISLRLFPAKPEEFPPITKCSWWIPSAAKVLALLNTANNLVHPPLIPYTDFYNSTLDHIDLMEEYHTWQNFGNSHRFSFCQYPFVISVAAKKIIIQRDSEQQMINIARQSLVDKVSRRQRPDMNILFLNMKVRRTHLVSDSLDELTRKRADLKKKLKVTFVGEAGLDMGGLTKEWFLLLIRQIFHPDYGMFTYHKDSHCHWFSSFKCDNYSEFRLVGILMGLAVYNSITLDIRFPPCCYKKLLSPPIIPSDQNIPVGICNVTVDDLCQIMPELAHGLSELLSHEGNVEEDFYSTFQVFQEEFGIIKSYNLKPGGDKISVTNQNRKEYVQLYTDFLLNKSIYKQFAAFYYGFHSVCASNALMLLRPEEVEILVCGSPDLDMHALQRSTQYDGYAKTDLTIKYFWDVVLGFPLDLQKKLLHFTTGSDRVPVGGMADLNFKISKNETSTNCLPVAHTCFNQLCLPPYKSKKDLKQKLIIGISNSEGFGLE",
# HECD3_HUMAN
"MAGPGPGAVLESPRQLLGRVRFLAEAARSLRAGRPLPAALAFVPREVLYKLYKDPAGPSRVLLPVWEAEGLGLRVGAAGPAPGTGSGPLRAARDSIELRRGACVRTTGEELCNGHGLWVKLTKEQLAEHLGDCGLQEGWLLVCRPAEGGARLVPIDTPNHLQRQQQLFGVDYRPVLRWEQVVDLTYSHRLGSRPQPAEAYAEAVQRLLYVPPTWTYECDEDLIHFLYDHLGKEDENLGSVKQYVESIDVSSYTEEFNVSCLTDSNADTYWESDGSQCQHWVRLTMKKGTIVKKLLLTVDTTDDNFMPKRVVVYGGEGDNLKKLSDVSIDETLIGDVCVLEDMTVHLPIIEIRIVECRDDGIDVRLRGVKIKSSRQRELGLNADLFQPTSLVRYPRLEGTDPEVLYRRAVLLQRFIKILDSVLHHLVPAWDHTLGTFSEIKQVKQFLLLSRQRPGLVAQCLRDSESSKPSFMPRLYINRRLAMEHRACPSRDPACKNAVFTQVYEGLKPSDKYEKPLDYRWPMRYDQWWECKFIAEGIIDQGGGFRDSLADMSEELCPSSADTPVPLPFFVRTANQGNGTGEARDMYVPNPSCRDFAKYEWIGQLMGAALRGKEFLVLALPGFVWKQLSGEEVSWSKDFPAVDSVLVKLLEVMEGMDKETFEFKFGKELTFTTVLSDQQVVELIPGGAGIVVGYGDRSRFIQLVQKARLEESKEQVAAMQAGLLKVVPQAVLDLLTWQELEKKVCGDPEVTVDALRKLTRFEDFEPSDSRVQYFWEALNNFTNEDRSRFLRFVTGRSRLPARIYIYPDKLGYETTDALPESSTCSSTLFLPHYASAKVCEEKLRYAAYNCVAIDTDMSPWEE",
# HECD4_HUMAN
"MPANHLPIGSTMSTVHLSSDGTYFYWIWSPASLNEKTPKGHSVFMDIFELVVENGVFVANPLQERTILMRKEGESAKSINEMLLSRLSRYRASPSATLAALTGSTISNTLKEDQAANTSCGLPLKMLRKTPIYTCGTYLVMLVPPPGGSGSSATRSLFGGTSGLSSLKRACYDTVNNMLWTCSNDYIDQWCNPGNQAFHYVCQRLGVSHIITEPKEEAITTNEVINQLLHHVGAMCIHQLNLLATNPNLPITSVLGKQHPIEAHHLSSICDIMEKAMVNGDTCIIRCILVVFQVVFKFFFSPQTERNRDIIRRSGLLLWQLLMAPKDQICPEIQKEVCLAISSGLNILYPGETEINNLLKLVLTEGERNSGLSQLRDVILTNLAEQLQNNRFGSDEDDHYRLNDELLHYILKIVVRESCILITKCQTVSKDDFQKLLSTVPAASSCLRYLMAVQNHLLSNTILIKPDENDDSDSSLQGETLKVQELKVSILALATQILTGCDEVLEMLQQVTTALINSDIADREQRLKGLEQVTKATMLGHLLPVLLTSLMHPNLQTLIMADALMPQLVQLVLYTSQTALLLKTQCPVFAEVGCSPCGAPDQKCRLFPDERMLEEKEEPGFLTGLKIPAPWAAGKTVETVHPVRDNYKFKETVHIPGARCLYLRFDSRCSSQYDYDKLVIYAGPNTNSRKVAEYGGNTLGYGSRSVLGTGWPKDLVKCISPPSLNFKVEGDTVTFSFEMRSGREHNTPDKAMWGFACTVRAQESSEDVSGGLPFLVDLALGLSVLACSMLRILYNGPEITKEEEACQELLRSKLLQRCQWQVEANGVISPALTPSPSPLPLTIEEDREFTYPSDVLVPPVGNYFDLPRIRLPPGIMIKLREISGRARPQFRPSIKEVIQPDVMEEMVVSCVIKHLNLVDALQSLINFQYQEEHAEEYDLLCKIMGETFKKLNAMERQLQSVAELEQKWQSEVDDAMQGKLENNMPFFYDYHFNENKMKELELLCSMKEVSFDGNDLENMVLSLREKFLQEVNSLIQKPSHPLAKTKTLVKSLMNRAELLLHVTIAAQSGLTRSISGTPAETPACKSASETKVISHAVRQPVFLRSMSAPSDLEMIGNEDLEFTRANQRRRHVTSHRSSSFTLLQSLAIEDSRDKPTYSVLLGQLFAFIGTNPDQAVSSSSFLLAAQTRWRRGNTRKQALVHMRELLTAAVRVGGVTHLVGPVTMVLQGGPRIEELTCGGMVEQVQEAFGETMTSVVSLCARYPIACANSIGLLCTIPYTRSEEKCLVRSGLVQLMDRLCSLSNQTESSSSEKQTKKQKVATMAWAAFQVLANRCVEWEKEEGGSTEAVHSGLARQVSSLLTNHLARATECCGNQAAGNDALQDVLSLLNDLSRSHIGKAILSQPACVSKLLSLLLDQRPSPKLVLIILQLCRAALPLMSVEDCGNVELPPWSYSVPSLNSEQEDPSDPASKIASLLLAKLADYVVPGCQTVLSPTASEPDTTLTKTSPKNSLKGDKDPGEESEAVDGKLSIFIHKREDQSSHEVLQPLLSSSEGRPFRLGTGANMEKVVKMDRDMTKGGCCEVITEEAAAALRKATKWAQSGLIVSIGPPVESINPETVSGLSTGDKKKTAQTSICRERNSELARTDPVRPFISGHVANSMAAEVIALLHSLLMAPESNAAQIWTTTAEKVLSRALMYIPQLGKYAESILENGSSSGRKLAKLQRIARQAVAALCALGGFKETIKIGSEVQVLGRGISGSIGVVASINEQEGIATVRFPPIDCRKTSQASDTLTIPLSRLCVPRSEALPLHKLSITEKVVQAVQSMLLPQEGSLSIHTSLPATGDGSAPVMAVVRLLAEIRTRACLVMAQLLEDSLFCEEFIQQCPAAVEVLNLVAQECSAGERLAVVEVQCERLRMLYRDCARPPPPPLQADRRQPKEITWSPSRVFPPVRACMFSSHLTSVTFLADPSAGGGLPRGTFIYATSPLPVQAPSFYWEIEIVSYGDTDDDTGPIVSFGFTTEAEKRDGAWTNPVGTCLFHNNGRAVHYNGSSLLQWKSVRLDVTLSPGDVAGIGWERTEGTPPPPGQPAKGRVYFTYCGQRLSPYLEDVSGGMWPVVHIQKKNTKTRANFGSRPFAYAEGQAHRNAADLCTDLAEEISANFEALPFAMASDSDNDAGTSIASDPGTHGPPCRIAAVATAQQQYDSDTSCHYKVELSYENFITSGPDPHPPPIADDESDDDDDDDIPQEDHYALLVKAWETKVFPTIRRRFRNEAERKSGLDQIKGALQLGMVDIARQTVEFLYEENGGIPRDLYLPTIEDIKDEANKFTIDKVRKGLTVVTRSPDSNNVASSAVGTALPKFAIRGMLKTFGLHGVVLDVDSVNELVQVETYLRSEGVLVRYWYPIDMLERPPAGYRRTATNGLVTLDNTNLQIHRELLRCEAALARLYCRMALLNIFAPKLPHLFTRLFHIPAIRDITLEHLQLLSNQLLAPPLPDGTISSSSILLAQSLQHCIHSQNCSATDLFYQGNSQTVREWLNVAITRTLHQGEESLLELTKQICSFLQTAPEQFPSEEFPISESKVNMDVNFPGAAFVVVSCKESQSGFRKDSSLYKAPWARVLVYGLGHKVKRNGQLNLIEAACYPRDASPANTGLAPPPTADQYPSVVLSTDRVHIKLGVSPPPGAVLVLHSLPLEFPLAMAFAEQLLSWKSEDSEGKSEDEPDTIPTSVLLQVVELLGNFLWTTDMAACVKELVFHLLAELLRTVHTLEQRRHPAGLSSSIALQLNPCLAMLMALQSELHKLYDEETQNWVSGGACGGSGGAAAGDQGRFSTYFHALMEGCLAVAEVTLPTNMSVTASGVTSATAPNLSDSSSSSSSSPGQTPQSPSLLSKRKKVKMKREKASSSGKRQSSRTVDSDPTVLSIGGSKPEDMLWFHRALTLLIILRHLTRKDPQGLGVTSDAIADACQALVGPTAHSRLLVISGIPTHLDEGVVRGAIRKACNAHGGVFKDEIYIPLQEEDTKKPKDKAEGGDGKVEPEKTLAFPGTDSMEVSTSSSLTPAMSISASASTSQASICSSQGISQTVSDLSVDPLPAGLELPIPPGLLEPHAVSSQESLDISLCSTGSLGSLGSLGEPLDNAETASVSDMGSMYTVTSLDNQPLAARPIKGFAVVEIRSRAKIEKIRASLFNNNDLIGLSSLDGEDELMEMSTEEILTVSVVNQSLFDTQGSPGLEDYFNDKSIKGEKLVPGAREVLTEIFKSCAHSEQTLSLTPAKPIRVSDIYLSKEQINSQTPGNLLHLFFTNVRPPKKVLEDQLTQILRKYGVPKPKFDKSKYSKAGKEQHPVKVVSTKRPITKPPAKDKAVLNSVSRTALSEKKPTVKPKSPEKSKPDEKDPEKSPTKKQEVPEEKYLTLEGFHKFVIDRARQDIRSVWRAILSCGYDLHFERCACIDVRHAQKASRKWTLEMDVALVQYINQLCRHLAITPARLHPHEVYLDPADAADPRVACLLNVPIESLRLRFALLQSLNTTLETFFLPLVELRQTPMYTHSIAALLKEAKGLIFYDTKVTVMNRVLNATVQRTADHAAPEITLDPLEIVGGEIRASENSYFCQAARQLASVPSSQLCVKLASGGDPTYAFNIRFTGEEVHGTSGSFRHFLWQVCKELQSSSLSLLLLCPSSAVNKNKGKYILTPSPITYGEEQLLHFLGQLLGIAIRADVPLPLDLLPSFWKTLVGEPLDPEQDLQEADILTYNYVKKFESINDETELEALCAEIASQHLATESPDSPNKPCCRFTYLTMTGEEVELCSRGRHILVAWENKDIYAAAIRSLRLRELQNVECVTAVRAGLGSIIPLQLLTMLSPLEMELRTCGLPYINLEFLKAHTMYQVGLMETDQHIEFFWGALEMFTQEELCKFIKFACNQERIPFTCPCKDGGPDTAHVPPYPMKIAPPDGTAGSPDSRYIRVETCMFMIKLPQYSSLEIMLEKLRCAIHYREDPLSG",
# HECW1_HUMAN
"MLLHLCSVKNLYQNRFLGLAAMASPSRNSQSRRRCKEPLRYSYNPDQFHNMDLRGGPHDGVTIPRSTSDTDLVTSDSRSTLMVSSSYYSIGHSQDLVIHWDIKEEVDAGDWIGMYLIDEVLSENFLDYKNRGVNGSHRGQIIWKIDASSYFVEPETKICFKYYHGVSGALRATTPSVTVKNSAAPIFKSIGADETVQGQGSRRLISFSLSDFQAMGLKKGMFFNPDPYLKISIQPGKHSIFPALPHHGQERRSKIIGNTVNPIWQAEQFSFVSLPTDVLEIEVKDKFAKSRPIIKRFLGKLSMPVQRLLERHAIGDRVVSYTLGRRLPTDHVSGQLQFRFEITSSIHPDDEEISLSTEPESAQIQDSPMNNLMESGSGEPRSEAPESSESWKPEQLGEGSVPDGPGNQSIELSRPAEEAAVITEAGDQGMVSVGPEGAGELLAQVQKDIQPAPSAEELAEQLDLGEEASALLLEDGEAPASTKEEPLEEEATTQSRAGREEEEKEQEEEGDVSTLEQGEGRLQLRASVKRKSRPCSLPVSELETVIASACGDPETPRTHYIRIHTLLHSMPSAQGGSAAEEEDGAEEESTLKDSSEKDGLSEVDTVAADPSALEEDREEPEGATPGTAHPGHSGGHFPSLANGAAQDGDTHPSTGSESDSSPRQGGDHSCEGCDASCCSPSCYSSSCYSTSCYSSSCYSASCYSPSCYNGNRFASHTRFSSVDSAKISESTVFSSQDDEEEENSAFESVPDSMQSPELDPESTNGAGPWQDELAAPSGHVERSPEGLESPVAGPSNRREGECPILHNSQPVSQLPSLRPEHHHYPTIDEPLPPNWEARIDSHGRVFYVDHVNRTTTWQRPTAAATPDGMRRSGSIQQMEQLNRRYQNIQRTIATERSEEDSGSQSCEQAPAGGGGGGGSDSEAESSQSSLDLRREGSLSPVNSQKITLLLQSPAVKFITNPEFFTVLHANYSAYRVFTSSTCLKHMILKVRRDARNFERYQHNRDLVNFINMFADTRLELPRGWEIKTDQQGKSFFVDHNSRATTFIDPRIPLQNGRLPNHLTHRQHLQRLRSYSAGEASEVSRNRGASLLARPGHSLVAAIRSQHQHESLPLAYNDKIVAFLRQPNIFEMLQERQPSLARNHTLREKIHYIRTEGNHGLEKLSCDADLVILLSLFEEEIMSYVPLQAAFHPGYSFSPRCSPCSSPQNSPGLQRASARAPSPYRRDFEAKLRNFYRKLEAKGFGQGPGKIKLIIRRDHLLEGTFNQVMAYSRKELQRNKLYVTFVGEEGLDYSGPSREFFFLLSQELFNPYYGLFEYSANDTYTVQISPMSAFVENHLEWFRFSGRILGLALIHQYLLDAFFTRPFYKALLRLPCDLSDLEYLDEEFHQSLQWMKDNNITDILDLTFTVNEEVFGQVTERELKSGGANTQVTEKNKKEYIERMVKWRVERGVVQQTEALVRGFYEVVDSRLVSVFDARELELVIAGTAEIDLNDWRNNTEYRGGYHDGHLVIRWFWAAVERFNNEQRLRLLQFVTGTSSVPYEGFAALRGSNGLRRFCIEKWGKITSLPRAHTCFNRLDLPPYPSYSMLYEKLLTAVEETSTFGLE",
# HECW2_HUMAN
"MASSAREHLLFVRRRNPQMRYTLSPENLQSLAAQSSMPENMTLQRANSDTDLVTSESRSSLTASMYEYTLGQAQNLIIFWDIKEEVDPSDWIGLYHIDENSPANFWDSKNRGVTGTQKGQIVWRIEPGPYFMEPEIKICFKYYHGISGALRATTPCITVKNPAVMMGAEGMEGGASGNLHSRKLVSFTLSDLRAVGLKKGMFFNPDPYLKMSIQPGKKSSFPTCAHHGQERRSTIISNTTNPIWHREKYSFFALLTDVLEIEIKDKFAKSRPIIKRFLGKLTIPVQRLLERQAIGDQMLSYNLGRRLPADHVSGYLQFKVEVTSSVHEDASPEAVGTILGVNSVNGDLGSPSDDEDMPGSHHDSQVCSNGPVSEDSAADGTPKHSFRTSSTLEIDTEELTSTSSRTSPPRGRQDSLNDYLDAIEHNGHSRPGTATCSERSMGASPKLRSSFPTDTRLNAMLHIDSDEEDHEFQQDLGYPSSLEEEGGLIMFSRASRADDGSLTSQTKLEDNPVENEEASTHEAASFEDKPENLPELAESSLPAGPAPEEGEGGPEPQPSADQGSAELCGSQEVDQPTSGADTGTSDASGGSRRAVSETESLDQGSEPSQVSSETEPSDPARTESVSEASTRPEGESDLECADSSCNESVTTQLSSVDTRCSSLESARFPETPAFSSQEEEDGACAAEPTSSGPAEGSQESVCTAGSLPVVQVPSGEDEGPGAESATVPDQEELGEVWQRRGSLEGAAAAAESPPQEEGSAGEAQGTCEGATAQEEGATGGSQANGHQPLRSLPSVRQDVSRYQRVDEALPPNWEARIDSHGRIFYVDHVNRTTTWQRPTAPPAPQVLQRSNSIQQMEQLNRRYQSIRRTMTNERPEENTNAIDGAGEEADFHQASADFRRENILPHSTSRSRITLLLQSPPVKFLISPEFFTVLHSNPSAYRMFTNNTCLKHMITKVRRDTHHFERYQHNRDLVGFLNMFANKQLELPRGWEMKHDHQGKAFFVDHNSRTTTFIDPRLPLQSSRPTSALVHRQHLTRQRSHSAGEVGEDSRHAGPPVLPRPSSTFNTVSRPQYQDMVPVAYNDKIVAFLRQPNIFEILQERQPDLTRNHSLREKIQFIRTEGTPGLVRLSSDADLVMLLSLFEEEIMSYVPPHALLHPSYCQSPRGSPVSSPQNSPGTQRANARAPAPYKRDFEAKLRNFYRKLETKGYGQGPGKLKLIIRRDHLLEDAFNQIMGYSRKDLQRNKLYVTFVGEEGLDYSGPSREFFFLVSRELFNPYYGLFEYSANDTYTVQISPMSAFVDNHHEWFRFSGRILGLALIHQYLLDAFFTRPFYKALLRILCDLSDLEYLDEEFHQSLQWMKDNDIHDILDLTFTVNEEVFGQITERELKPGGANIPVTEKNKKEYIERMVKWRIERGVVQQTESLVRGFYEVVDARLVSVFDARELELVIAGTAEIDLSDWRNNTEYRGGYHDNHIVIRWFWAAVERFNNEQRLRLLQFVTGTSSIPYEGFASLRGSNGPRRFCVEKWGKITALPRAHTCFNRLDLPPYPSFSMLYEKLLTAVEETSTFGLE",
# HERC1_HUMAN
"MATMIPPVKLKWLEHLNSSWITEDSESIATREGVAVLYSKLVSNKEVVPLPQQVLCLKGPQLPDFERESLSSDEQDHYLDALLSSQLALAKMVCSDSPFAGALRKRLLVLQRVFYALSNKYHDKGKVKQQQHSPESSSGSADVHSVSERPRSSTDALIEMGVRTGLSLLFALLRQSWMMPVSGPGLSLCNDVIHTAIEVVSSLPPLSLANESKIPPMGLDCLSQVTTFLKGVTIPNSGADTLGRRLASELLLGLAAQRGSLRYLLEWIEMALGASAVVHTMEKGKLLSSQEGMISFDCFMTILMQMRRSLGSSADRSQWREPTRTSDGLCSLYEAALCLFEEVCRMASDYSRTCASPDSIQTGDAPIVSETCEVYVWGSNSSHQLVEGTQEKILQPKLAPSFSDAQTIEAGQYCTFVISTDGSVRACGKGSYGRLGLGDSNNQSTLKKLTFEPHRSIKKVSSSKGSDGHTLAFTTEGEVFSWGDGDYGKLGHGNSSTQKYPKLIQGPLQGKVVVCVSAGYRHSAAVTEDGELYTWGEGDFGRLGHGDSNSRNIPTLVKDISNVGEVSCGSSHTIALSKDGRTVWSFGGGDNGKLGHGDTNRVYKPKVIEALQGMFIRKVCAGSQSSLALTSTGQVYAWGCGACLGCGSSEATALRPKLIEELAATRIVDVSIGDSHCLALSHDNEVYAWGNNSMGQCGQGNSTGPITKPKKVSGLDGIAIQQISAGTSHSLAWTALPRDRQVVAWHRPYCVDLEESTFSHLRSFLERYCDKINSEIPPLPFPSSREHHSFLKLCLKLLSNHLALALAGGVATSILGRQAGPLRNLLFRLMDSTVPDEIQEVVIETLSVGATMLLPPLRERMELLHSLLPQGPDRWESLSKGQRMQLDIILTSLQDHTHVASLLGYSSPSDAADLSSVCTGYGNLSDQPYGTQSCHPDTHLAEILMKTLLRNLGFYTDQAFGELEKNSDKFLLGTSSSENSQPAHLHELLCSLQKQLLAFCHINNISENSSSVALLHKHLQLLLPHATDIYSRSANLLKESPWNGSVGEKLRDVIYVSAAGSMLCQIVNSLLLLPVSVARPLLSYLLDLLPPLDCLNRLLPAADLLEDQELQWPLHGGPELIDPAGLPLPQPAQSWVWLVDLERTIALLIGRCLGGMLQGSPVSPEEQDTAYWMKTPLFSDGVEMDTPQLDKCMSCLLEVALSGNEEQKPFDYKLRPEIAVYVDLALGCSKEPARSLWISMQDYAVSKDWDSATLSNESLLDTVSRFVLAALLKHTNLLSQACGESRYQPGKHLSEVYRCVYKVRSRLLACKNLELIQTRSSSRDRWISENQDSADVDPQEHSFTRTIDEEAEMEEQAERDREEGHPEPEDEEEEREHEVMTAGKIFQCFLSAREVARSRDRDRMNSGAGSGARADDPPPQSQQERRVSTDLPEGQDVYTAACNSVIHRCALLILGVSPVIDELQKRREEGQLQQPSTSASEGGGLMTRSESLTAESRLVHTSPNYRLIKSRSESDLSQPESDEEGYALSGRRNVDLDLAASHRKRGPMHSQLESLSDSWARLKHSRDWLCNSSYSFESDFDLTKSLGVHTLIENVVSFVSGDVGNAPGFKEPEESMSTSPQASIIAMEQQQLRAELRLEALHQILVLLSGMEEKGSISLAGSRLSSGFQSSTLLTSVRLQFLAGCFGLGTVGHTGGKGESGRLHHYQDGIRAAKRNIQIEIQVAVHKIYQQLSATLERALQANKHHIEAQQRLLLVTVFALSVHYQPVDVSLAISTGLLNVLSQLCGTDTMLGQPLQLLPKTGVSQLSTALKVASTRLLQILAITTGTYADKLSPKVVQSLLDLLCSQLKNLLSQTGVLHMASFGEGEQEDGEEEEKKVDSSGETEKKDFRAALRKQHAAELHLGDFLVFLRRVVSSKAIQSKMASPKWTEVLLNIASQKCSSGIPLVGNLRTRLLALHVLEAVLPACESGVEDDQMAQIVERLFSLLSDCMWETPIAQAKHAIQIKEKEQEIKLQKQGELEEEDENLPIQEVSFDPEKAQCCLVENGQILTHGSGGKGYGLASTGVTSGCYQWKFYIVKENRGNEGTCVGVSRWPVHDFNHRTTSDMWLYRAYSGNLYHNGEQTLTLSSFTQGDFITCVLDMEARTISFGKNGEEPKLAFEDVDAAELYPCVMFYSSNPGEKVKICDMQMRGTPRDLLPGDPICSPVAAVLAEATIQLIRILHRTDRWTYCINKKMMERLHKIKICIKESGQKLKKSRSVQSREENEMREEKESKEEEKGKHTRHGLADLSELQLRTLCIEVWPVLAVIGGVDAGLRVGGRCVHKQTGRHATLLGVVKEGSTSAKVQWDEAEITISFPTFWSPSDTPLYNLEPCEPLPFDVARFRGLTASVLLDLTYLTGVHEDMGKQSTKRHEKKHRHESEEKGDVEQKPESESALDMRTGLTSDDVKSQSTTSSKSENEIASFSLDPTLPSVESQHQITEGKRKNHEHMSKNHDVAQSEIRAVQLSYLYLGAMKSLSALLGCSKYAELLLIPKVLAENGHNSDCASSPVVHEDVEMRAALQFLMRHMVKRAVMRSPIKRALGLADLERAQAMIYKLVVHGLLEDQFGGKIKQEIDQQAEESDPAQQAQTPVTTSPSASSTTSFMSSSLEDTTTATTPVTDTETVPASESPGVMPLSLLRQMFSSYPTTTVLPTRRAQTPPISSLPTSPSDEVGRRQSLTSPDSQSARPANRTALSDPSSRLSTSPPPPAIAVPLLEMGFSLRQIAKAMEATGARGEADAQNITVLAMWMIEHPGHEDEEEPQSGSTADSRPGAAVLGSGGKSNDPCYLQSPGDIPSADAAEMEEGFSESPDNLDHTENAASGSGPSARGRSAVTRRHKFDLAARTLLARAAGLYRSVQAHRNQSRREGISLQQDPGALYDFNLDEELEIDLDDEAMEAMFGQDLTSDNDILGMWIPEVLDWPTWHVCESEDREEVVVCELCECSVVSFNQHMKRNHPGCGRSANRQGYRSNGSYVDGWFGGECGSGNPYYLLCGTCREKYLAMKTKSKSTSSERYKGQAPDLIGKQDSVYEEDWDMLDVDEDEKLTGEEEFELLAGPLGLNDRRIVPEPVQFPDSDPLGASVAMVTATNSMEETLMQIGCHGSVEKSSSGRITLGEQAAALANPHDRVVALRRVTAAAQVLLARTMVMRALSLLSVSGSSCSLAAGLESLGLTDIRTLVRLMCLAAAGRAGLSTSPSAMASTSERSRGGHSKANKPISCLAYLSTAVGCLASNAPSAAKLLVQLCTQNLISAATGVNLTTVDDSIQRKFLPSFLRGIAEENKLVTSPNFVVTQALVALLADKGAKLRPNYDKSEVEKKGPLELANALAACCLSSRLSSQHRQWAAQQLVRTLAAHDRDNQTTLQTLADMGGDLRKCSFIKLEAHQNRVMTCVWCNKKGLLATSGNDGTIRVWNVTKKQYSLQQTCVFNRLEGDAEESLGSPSDPSFSPVSWSISGKYLAGALEKMVNIWQVNGGKGLVDIQPHWVSALAWPEEGPATAWSGESPELLLVGRMDGSLGLIEVVDVSTMHRRELEHCYRKDVSVTCIAWFSEDRPFAVGYFDGKLLLGTKEPLEKGGIVLIDAHKDTLISMKWDPTGHILMTCAKEDSVKLWGSISGCWCCLHSLCHPSIVNGIAWCRLPGKGSKLQLLMATGCQSGLVCVWRIPQDTTQTNVTSAEGWWEQESNCQDGYRKSSGAKCVYQLRGHITPVRTVAFSSDGLALVSGGLGGLMNIWSLRDGSVLQTVVIGSGAIQTTVWIPEVGVAACSNRSKDVLVVNCTAEWAAANHVLATCRTALKQQGVLGLNMAPCMRAFLERLPMMLQEQYAYEKPHVVCGDQLVHSPYMQCLASLAVGLHLDQLLCNPPVPPHHQNCLPDPASWNPNEWAWLECFSTTIKAAEALTNGAQFPESFTVPDLEPVPEDELVFLMDNSKWINGMDEQIMSWATSRPEDWHLGGKCDVYLWGAGRHGQLAEAGRNVMVPAAAPSFSQAQQVICGQNCTFVIQANGTVLACGEGSYGRLGQGNSDDLHVLTVISALQGFVVTQLVTSCGSDGHSMALTESGEVFSWGDGDYGKLGHGNSDRQRRPRQIEALQGEEVVQMSCGFKHSAVVTSDGKLFTFGNGDYGRLGLGNTSNKKLPERVTALEGYQIGQVACGLNHTLAVSADGSMVWAFGDGDYGKLGLGNSTAKSSPQKIDVLCGIGIKKVACGTQFSVALTKDGHVYTFGQDRLIGLPEGRARNHNRPQQIPVLAGVIIEDVAVGAEHTLALASNGDVYAWGSNSEGQLGLGHTNHVREPTLVTGLQGKNVRQISAGRCHSAAWTAPPVPPRAPGVSVPLQLGLPDTVPPQYGALREVSIHTVRARLRLLYHFSDLMYSSWRLLNLSPNNQNSTSHYNAGTWGIVQGQLRPLLAPRVYTLPMVRSIGKTMVQGKNYGPQITVKRISTRGRKCKPIFVQIARQVVKLNASDLRLPSRAWKVKLVGEGADDAGGVFDDTITEMCQELETGIVDLLIPSPNATAEVGYNRDRFLFNPSACLDEHLMQFKFLGILMGVAIRTKKPLDLHLAPLVWKQLCCVPLTLEDLEEVDLLYVQTLNSILHIEDSGITEESFHEMIPLDSFVGQSADGKMVPIIPGGNSIPLTFSNRKEYVERAIEYRLHEMDRQVAAVREGMSWIVPVPLLSLLTAKQLEQMVCGMPEISVEVLKKVVRYREVDEQHQLVQWFWHTLEEFSNEERVLFMRFVSGRSRLPANTADISQRFQIMKVDRPYDSLPTSQTCFFQLRLPPYSSQLVMAERLRYAINNCRSIDMDNYMLSRNVDNAEGSDTDY",
# HERC2_HUMAN
"MPSESFCLAAQARLDSKWLKTDIQLAFTRDGLCGLWNEMVKDGEIVYTGTESTQNGELPPRKDDSVEPSGTKKEDLNDKEKKDEEETPAPIYRAKSILDSWVWGKQPDVNELKECLSVLVKEQQALAVQSATTTLSALRLKQRLVILERYFIALNRTVFQENVKVKWKSSGISLPPVDKKSSRPAGKGVEGLARVGSRAALSFAFAFLRRAWRSGEDADLCSELLQESLDALRALPEASLFDESTVSSVWLEVVERATRFLRSVVTGDVHGTPATKGPGSIPLQDQHLALAILLELAVQRGTLSQMLSAILLLLQLWDSGAQETDNERSAQGTSAPLLPLLQRFQSIICRKDAPHSEGDMHLLSGPLSPNESFLRYLTLPQDNELAIDLRQTAVVVMAHLDRLATPCMPPLCSSPTSHKGSLQEVIGWGLIGWKYYANVIGPIQCEGLANLGVTQIACAEKRFLILSRNGRVYTQAYNSDTLAPQLVQGLASRNIVKIAAHSDGHHYLALAATGEVYSWGCGDGGRLGHGDTVPLEEPKVISAFSGKQAGKHVVHIACGSTYSAAITAEGELYTWGRGNYGRLGHGSSEDEAIPMLVAGLKGLKVIDVACGSGDAQTLAVTENGQVWSWGDGDYGKLGRGGSDGCKTPKLIEKLQDLDVVKVRCGSQFSIALTKDGQVYSWGKGDNQRLGHGTEEHVRYPKLLEGLQGKKVIDVAAGSTHCLALTEDSEVHSWGSNDQCQHFDTLRVTKPEPAALPGLDTKHIVGIACGPAQSFAWSSCSEWSIGLRVPFVVDICSMTFEQLDLLLRQVSEGMDGSADWPPPQEKECVAVATLNLLRLQLHAAISHQVDPEFLGLGLGSILLNSLKQTVVTLASSAGVLSTVQSAAQAVLQSGWSVLLPTAEERARALSALLPCAVSGNEVNISPGRRFMIDLLVGSLMADGGLESALHAAITAEIQDIEAKKEAQKEKEIDEQEANASTFHRSRTPLDKDLINTGICESSGKQCLPLVQLIQQLLRNIASQTVARLKDVARRISSCLDFEQHSRERSASLDLLLRFQRLLISKLYPGESIGQTSDISSPELMGVGSLLKKYTALLCTHIGDILPVAASIASTSWRHFAEVAYIVEGDFTGVLLPELVVSIVLLLSKNAGLMQEAGAVPLLGGLLEHLDRFNHLAPGKERDDHEELAWPGIMESFFTGQNCRNNEEVTLIRKADLENHNKDGGFWTVIDGKVYDIKDFQTQSLTGNSILAQFAGEDPVVALEAALQFEDTRESMHAFCVGQYLEPDQEIVTIPDLGSLSSPLIDTERNLGLLLGLHASYLAMSTPLSPVEIECAKWLQSSIFSGGLQTSQIHYSYNEEKDEDHCSSPGGTPASKSRLCSHRRALGDHSQAFLQAIADNNIQDHNVKDFLCQIERYCRQCHLTTPIMFPPEHPVEEVGRLLLCCLLKHEDLGHVALSLVHAGALGIEQVKHRTLPKSVVDVCRVVYQAKCSLIKTHQEQGRSYKEVCAPVIERLRFLFNELRPAVCNDLSIMSKFKLLSSLPRWRRIAQKIIRERRKKRVPKKPESTDDEEKIGNEESDLEEACILPHSPINVDKRPIAIKSPKDKWQPLLSTVTGVHKYKWLKQNVQGLYPQSPLLSTIAEFALKEEPVDVEKMRKCLLKQLERAEVRLEGIDTILKLASKNFLLPSVQYAMFCGWQRLIPEGIDIGEPLTDCLKDVDLIPPFNRMLLEVTFGKLYAWAVQNIRNVLMDASAKFKELGIQPVPLQTITNENPSGPSLGTIPQARFLLVMLSMLTLQHGANNLDLLLNSGMLALTQTALRLIGPSCDNVEEDMNASAQGASATVLEETRKETAPVQLPVSGPELAAMMKIGTRVMRGVDWKWGDQDGPPPGLGRVIGELGEDGWIRVQWDTGSTNSYRMGKEGKYDLKLAELPAAAQPSAEDSDTEDDSEAEQTERNIHPTAMMFTSTINLLQTLCLSAGVHAEIMQSEATKTLCGLLRMLVESGTTDKTSSPNRLVYREQHRSWCTLGFVRSIALTPQVCGALSSPQWITLLMKVVEGHAPFTATSLQRQILAVHLLQAVLPSWDKTERARDMKCLVEKLFDFLGSLLTTCSSDVPLLRESTLRRRRVRPQASLTATHSSTLAEEVVALLRTLHSLTQWNGLINKYINSQLRSITHSFVGRPSEGAQLEDYFPDSENPEVGGLMAVLAVIGGIDGRLRLGGQVMHDEFGEGTVTRITPKGKITVQFSDMRTCRVCPLNQLKPLPAVAFNVNNLPFTEPMLSVWAQLVNLAGSKLEKHKIKKSTKQAFAGQVDLDLLRCQQLKLYILKAGRALLSHQDKLRQILSQPAVQETGTVHTDDGAVVSPDLGDMSPEGPQPPMILLQQLLASATQPSPVKAIFDKQELEAAALAVCQCLAVESTHPSSPGFEDCSSSEATTPVAVQHIRPARVKRRKQSPVPALPIVVQLMEMGFSRRNIEFALKSLTGASGNASSLPGVEALVGWLLDHSDIQVTELSDADTVSDEYSDEEVVEDVDDAAYSMSTGAVVTESQTYKKRADFLSNDDYAVYVRENIQVGMMVRCCRAYEEVCEGDVGKVIKLDRDGLHDLNVQCDWQQKGGTYWVRYIHVELIGYPPPSSSSHIKIGDKVRVKASVTTPKYKWGSVTHQSVGVVKAFSANGKDIIVDFPQQSHWTGLLSEMELVPSIHPGVTCDGCQMFPINGSRFKCRNCDDFDFCETCFKTKKHNTRHTFGRINEPGQSAVFCGRSGKQLKRCHSSQPGMLLDSWSRMVKSLNVSSSVNQASRLIDGSEPCWQSSGSQGKHWIRLEIFPDVLVHRLKMIVDPADSSYMPSLVVVSGGNSLNNLIELKTININPSDTTVPLLNDCTEYHRYIEIAIKQCRSSGIDCKIHGLILLGRIRAEEEDLAAVPFLASDNEEEEDEKGNSGSLIRKKAAGLESAATIRTKVFVWGLNDKDQLGGLKGSKIKVPSFSETLSALNVVQVAGGSKSLFAVTVEGKVYACGEATNGRLGLGISSGTVPIPRQITALSSYVVKKVAVHSGGRHATALTVDGKVFSWGEGDDGKLGHFSRMNCDKPRLIEALKTKRIRDIACGSSHSAALTSSGELYTWGLGEYGRLGHGDNTTQLKPKMVKVLLGHRVIQVACGSRDAQTLALTDEGLVFSWGDGDFGKLGRGGSEGCNIPQNIERLNGQGVCQIECGAQFSLALTKSGVVWTWGKGDYFRLGHGSDVHVRKPQVVEGLRGKKIVHVAVGALHCLAVTDSGQVYAWGDNDHGQQGNGTTTVNRKPTLVQGLEGQKITRVACGSSHSVAWTTVDVATPSVHEPVLFQTARDPLGASYLGVPSDADSSAASNKISGASNSKPNRPSLAKILLSLDGNLAKQQALSHILTALQIMYARDAVVGALMPAAMIAPVECPSFSSAAPSDASAMASPMNGEECMLAVDIEDRLSPNPWQEKREIVSSEDAVTPSAVTPSAPSASARPFIPVTDDLGAASIIAETMTKTKEDVESQNKAAGPEPQALDEFTSLLIADDTRVVVDLLKLSVCSRAGDRGRDVLSAVLSGMGTAYPQVADMLLELCVTELEDVATDSQSGRLSSQPVVVESSHPYTDDTSTSGTVKIPGAEGLRVEFDRQCSTERRHDPLTVMDGVNRIVSVRSGREWSDWSSELRIPGDELKWKFISDGSVNGWGWRFTVYPIMPAAGPKELLSDRCVLSCPSMDLVTCLLDFRLNLASNRSIVPRLAASLAACAQLSALAASHRMWALQRLRKLLTTEFGQSININRLLGENDGETRALSFTGSALAALVKGLPEALQRQFEYEDPIVRGGKQLLHSPFFKVLVALACDLELDTLPCCAETHKWAWFRRYCMASRVAVALDKRTPLPRLFLDEVAKKIRELMADSENMDVLHESHDIFKREQDEQLVQWMNRRPDDWTLSAGGSGTIYGWGHNHRGQLGGIEGAKVKVPTPCEALATLRPVQLIGGEQTLFAVTADGKLYATGYGAGGRLGIGGTESVSTPTLLESIQHVFIKKVAVNSGGKHCLALSSEGEVYSWGEAEDGKLGHGNRSPCDRPRVIESLRGIEVVDVAAGGAHSACVTAAGDLYTWGKGRYGRLGHSDSEDQLKPKLVEALQGHRVVDIACGSGDAQTLCLTDDDTVWSWGDGDYGKLGRGGSDGCKVPMKIDSLTGLGVVKVECGSQFSVALTKSGAVYTWGKGDYHRLGHGSDDHVRRPRQVQGLQGKKVIAIATGSLHCVCCTEDGEVYTWGDNDEGQLGDGTTNAIQRPRLVAALQGKKVNRVACGSAHTLAWSTSKPASAGKLPAQVPMEYNHLQEIPIIALRNRLLLLHHLSELFCPCIPMFDLEGSLDETGLGPSVGFDTLRGILISQGKEAAFRKVVQATMVRDRQHGPVVELNRIQVKRSRSKGGLAGPDGTKSVFGQMCAKMSSFGPDSLLLPHRVWKVKFVGESVDDCGGGYSESIAEICEELQNGLTPLLIVTPNGRDESGANRDCYLLSPAARAPVHSSMFRFLGVLLGIAIRTGSPLSLNLAEPVWKQLAGMSLTIADLSEVDKDFIPGLMYIRDNEATSEEFEAMSLPFTVPSASGQDIQLSSKHTHITLDNRAEYVRLAINYRLHEFDEQVAAVREGMARVVPVPLLSLFTGYELETMVCGSPDIPLHLLKSVATYKGIEPSASLIQWFWEVMESFSNTERSLFLRFVWGRTRLPRTIADFRGRDFVIQVLDKYNPPDHFLPESYTCFFLLKLPRYSCKQVLEEKLKYAIHFCKSIDTDDYARIALTGEPAADDSSDDSDNEDVDSFASDSTQDYLTGH",
# HERC3_HUMAN
"MLCWGYWSLGQPGISTNLQGIVAEPQVCGFISDRSVKEVACGGNHSVFLLEDGEVYTCGLNTKGQLGHEREGNKPEQIGALADQHIIHVACGESHSLALSDRGQLFSWGAGSDGQLGLMTTEDSVAVPRLIQKLNQQTILQVSCGNWHCLALAADGQFFTWGKNSHGQLGLGKEFPSQASPQRVRSLEGIPLAQVAAGGAHSFALSLSGAVFGWGMNNAGQLGLSDEKDRESPCHVKLLRTQKVVYISCGEEHTAVLTKSGGVFTFGAGSCGQLGHDSMNDEVNPRRVLELMGSEVTQIACGRQHTLAFVPSSGLIYAFGCGARGQLGTGHTCNVKCPSPVKGYWAAHSGQLSARADRFKYHIVKQIFSGGDQTFVLCSKYENYSPAVDFRTMNQAHYTSLINDETIAVWRQKLSEHNNANTINGVVQILSSAACWNGSFLEKKIDEHFKTSPKIPGIDLNSTRVLFEKLMNSQHSMILEQILNSFESCLIPQLSSSPPDVEAMRIYLILPEFPLLQDSKYYITLTIPLAMAILRLDTNPSKVLDNWWSQVCPKYFMKLVNLYKGAVLYLLRGRKTFLIPVLFNNYITAALKLLEKLYKVNLKVKHVEYDTFYIPEISNLVDIQEDYLMWFLHQAGMKARPSIIQDTVTLCSYPFIFDAQAKTKMLQTDAELQMQVAVNGANLQNVFMLLTLEPLLARSPFLVLHVRRNNLVGDALRELSIHSDIDLKKPLKVIFDGEEAVDAGGVTKEFFLLLLKELLNPIYGMFTYYQDSNLLWFSDTCFVEHNWFHLIGITCGLAIYNSTVVDLHFPLALYKKLLNVKPGLEDLKELSPTEGRSLQELLDYPGEDVEETFCLNFTICRESYGVIEQKKLIPGGDNVTVCKDNRQEFVDAYVNYVFQISVHEWYTAFSSGFLKVCGGKVLELFQPSELRAMMVGNSNYNWEELEETAIYKGDYSATHPTVKLFWETFHEFPLEKKKKFLLFLTGSDRIPIYGMASLQIVIQSTASGEEYLPVAHTCYNLLDLPKYSSKEILSARLTQALDNYEGFSLA",
# HERC4_HUMAN
"MLCWGNASFGQLGLGGIDEEIVLEPRKSDFFINKRVRDVGCGLRHTVFVLDDGTVYTCGCNDLGQLGHEKSRKKPEQVVALDAQNIVAVSCGEAHTLALNDKGQVYAWGLDSDGQLGLVGSEECIRVPRNIKSLSDIQIVQVACGYYHSLALSKASEVFCWGQNKYGQLGLGTDCKKQTSPQLLKSLLGIPFMQVAAGGAHSFVLTLSGAIFGWGRNKFGQLGLNDENDRYVPNLLKSLRSQKIVYICCGEDHTAALTKEGGVFTFGAGGYGQLGHNSTSHEINPRKVFELMGSIVTEIACGRQHTSAFVPSSGRIYSFGLGGNGQLGTGSTSNRKSPFTVKGNWYPYNGQCLPDIDSEEYFCVKRIFSGGDQSFSHYSSPQNCGPPDDFRCPNPTKQIWTVNEALIQKWLSYPSGRFPVEIANEIDGTFSSSGCLNGSFLAVSNDDHYRTGTRFSGVDMNAARLLFHKLIQPDHPQISQQVAASLEKNLIPKLTSSLPDVEALRFYLTLPECPLMSDSNNFTTIAIPFGTALVNLEKAPLKVLENWWSVLEPPLFLKIVELFKEVVVHLLKLYKIGIPPSERRIFNSFLHTALKVLEILHRVNEKMGQIIQYDKFYIHEVQELIDIRNDYINWVQQQAYGMDVNHGLTELADIPVTICTYPFVFDAQAKTTLLQTDAVLQMQMAIDQAHRQNVSSLFLPVIESVNPCLILVVRRENIVGDAMEVLRKTKNIDYKKPLKVIFVGEDAVDAGGVRKEFFLLIMRELLDPKYGMFRYYEDSRLIWFSDKTFEDSDLFHLIGVICGLAIYNCTIVDLHFPLALYKKLLKKKPSLDDLKELMPDVGRSMQQLLDYPEDDIEETFCLNFTITVENFGATEVKELVLNGADTAVNKQNRQEFVDAYVDYIFNKSVASLFDAFHAGFHKVCGGKVLLLFQPNELQAMVIGNTNYDWKELEKNTEYKGEYWAEHPTIKIFWEVFHELPLEKKKQFLLFLTGSDRIPILGMKSLKLVIQSTGGGEEYLPVSHTCFNLLDLPKYTEKETLRSKLIQAIDHNEGFSLI",
# HERC5_HUMAN
"MERRSRRKSRRNGRSTAGKAAATQPAKSPGAQLWLFPSAAGLHRALLRRVEVTRQLCCSPGRLAVLERGGAGVQVHQLLAGSGGARTPKCIKLGKNMKIHSVDQGAEHMLILSSDGKPFEYDNYSMKHLRFESILQEKKIIQITCGDYHSLALSKGGELFAWGQNLHGQLGVGRKFPSTTTPQIVEHLAGVPLAQISAGEAHSMALSMSGNIYSWGKNECGQLGLGHTESKDDPSLIEGLDNQKVEFVACGGSHSALLTQDGLLFTFGAGKHGQLGHNSTQNELRPCLVAELVGYRVTQIACGRWHTLAYVSDLGKVFSFGSGKDGQLGNGGTRDQLMPLPVKVSSSEELKLESHTSEKELIMIAGGNQSILLWIKKENSYVNLKRTIPTLNEGTVKRWIADVETKRWQSTKREIQEIFSSPACLTGSFLRKRRTTEMMPVYLDLNKARNIFKELTQKDWITNMITTCLKDNLLKRLPFHSPPQEALEIFFLLPECPMMHISNNWESLVVPFAKVVCKMSDQSSLVLEEYWATLQESTFSKLVQMFKTAVICQLDYWDESAEENGNVQALLEMLKKLHRVNQVKCQLPESIFQVDELLHRLNFFVEVCRRYLWKMTVDASENVQCCVIFSHFPFIFNNLSKIKLLHTDTLLKIESKKHKAYLRSAAIEEERESEFALRPTFDLTVRRNHLIEDVLNQLSQFENEDLRKELWVSFSGEIGYDLGGVKKEFFYCLFAEMIQPEYGMFMYPEGASCMWFPVKPKFEKKRYFFFGVLCGLSLFNCNVANLPFPLALFKKLLDQMPSLEDLKELSPDLGKNLQTLLDDEGDNFEEVFYIHFNVHWDRNDTNLIPNGSSITVNQTNKRDYVSKYINYIFNDSVKAVYEEFRRGFYKMCDEDIIKLFHPEELKDVIVGNTDYDWKTFEKNARYEPGYNSSHPTIVMFWKAFHKLTLEEKKKFLVFLTGTDRLQMKDLNNMKITFCCPESWNERDPIRALTCFSVLFLPKYSTMETVEEALQEAINNNRGFG",
# HERC6_HUMAN
"MYFCWGADSRELQRRRTAGSPGAELLQAASGERHSLLLLTNHRVLSCGDNSRGQLGRRGAQRGELPEPIQALETLIVDLVSCGKEHSLAVCHKGRVFAWGAGSEGQLGIGEFKEISFTPKKIMTLNDIKIIQVSCGHYHSLALSKDSQVFSWGKNSHGQLGLGKEFPSQASPQRVRSLEGIPLAQVAAGGAHSFALSLCGTSFGWGSNSAGQLALSGRNVPVQSNKPLSVGALKNLGVVYISCGDAHTAVLTQDGKVFTFGDNRSGQLGYSPTPEKRGPQLVERIDGLVSQIDCGSYHTLAYVHTTGQVVSFGHGPSDTSKPTHPEALTENFDISCLISAEDFVDVQVKHIFAGTYANFVTTHQDTSSTRAPGKTLPEISRISQSMAEKWIAVKRRSTEHEMAKSEIRMIFSSPACLTASFLKKRGTGETTSIDVDLEMARDTFKKLTKKEWISSMITTCLEDDLLRALPCHSPHQEALSVFLLLPECPVMHDSKNWKNLVVPFAKAVCEMSKQSLQVLKKCWAFLQESSLNPLIQMLKAAIISQLLHQTKTEQDHCNVKALLGMMKELHKVNKANCRLPENTFNINELSNLLNFYIDRGRQLFRDNHLIPAETPSPVIFSDFPFIFNSLSKIKLLQADSHIKMQMSEKKAYMLMHETILQKKDEFPPSPRFILRVRRSRLVKDALRQLSQAEATDFCKVLVVEFINEICPESGGVSSEFFHCMFEEMTKPEYGMFMYPEMGSCMWFPAKPKPEKKRYFLFGMLCGLSLFNLNVANLPFPLALYKKLLDQKPSLEDLKELSPRLGKSLQEVLDDAADDIGDALCIRFSIHWDQNDVDLIPNGISIPVDQTNKRDYVSKYIDYIFNVSVKAVYEEFQRGFYRVCEKEILRHFYPEELMTAIIGNTDYDWKQFEQNSKYEQGYQKSHPTIQLFWKAFHKLTLDEKKKFLFFLTGRDRLHARGIQKMEIVFRCPETFSERDHPTSITCHNILSLPKYSTMERMEEALQVAINNNRGFVSPMLTQS",
# HLTF_HUMAN
"MSWMFKRDPVWKYLQTVQYGVHGNFPRLSYPTFFPRFEFQDVIPPDDFLTSDEEVDSVLFGSLRGHVVGLRYYTGVVNNNEMVALQRDPNNPYDKNAIKVNNVNGNQVGHLKKELAGALAYIMDNKLAQIEGVVPFGANNAFTMPLHMTFWGKEENRKAVSDQLKKHGFKLGPAPKTLGFNLESGWGSGRAGPSYSMPVHAAVQMTTEQLKTEFDKLFEDLKEDDKTHEMEPAEAIETPLLPHQKQALAWMVSRENSKELPPFWEQRNDLYYNTITNFSEKDRPENVHGGILADDMGLGKTLTAIAVILTNFHDGRPLPIERVKKNLLKKEYNVNDDSMKLGGNNTSEKADGLSKDASRCSEQPSISDIKEKSKFRMSELSSSRPKRRKTAVQYIESSDSEEIETSELPQKMKGKLKNVQSETKGRAKAGSSKVIEDVAFACALTSSVPTTKKKMLKKGACAVEGSKKTDVEERPRTTLIICPLSVLSNWIDQFGQHIKSDVHLNFYVYYGPDRIREPALLSKQDIVLTTYNILTHDYGTKGDSPLHSIRWLRVILDEGHAIRNPNAQQTKAVLDLESERRWVLTGTPIQNSLKDLWSLLSFLKLKPFIDREWWHRTIQRPVTMGDEGGLRRLQSLIKNITLRRTKTSKIKGKPVLELPERKVFIQHITLSDEERKIYQSVKNEGRATIGRYFNEGTVLAHYADVLGLLLRLRQICCHTYLLTNAVSSNGPSGNDTPEELRKKLIRKMKLILSSGSDEECAICLDSLTVPVITHCAHVFCKPCICQVIQNEQPHAKCPLCRNDIHEDNLLECPPEELARDSEKKSDMEWTSSSKINALMHALTDLRKKNPNIKSLVVSQFTTFLSLIEIPLKASGFVFTRLDGSMAQKKRVESIQCFQNTEAGSPTIMLLSLKAGGVGLNLSAASRVFLMDPAWNPAAEDQCFDRCHRLGQKQEVIITKFIVKDSVEENMLKIQNKKRELAAGAFGTKKPNADEMKQAKINEIRTLIDL",
# HUWE1_HUMAN
"MKVDRTKLKKTPTEAPADCRALIDKLKVCNDEQLLLELQQIKTWNIGKCELYHWVDLLDRFDGILADAGQTVENMSWMLVCDRPEREQLKMLLLAVLNFTALLIEYSFSRHLYSSIEHLTTLLASSDMQVVLAVLNLLYVFSKRSNYITRLGSDKRTPLLTRLQHLAESWGGKENGFGLAECCRDLHMMKYPPSATTLHFEFYADPGAEVKIEKRTTSNTLHYIHIEQLDKISESPSEIMESLTKMYSIPKDKQMLLFTHIRLAHGFSNHRKRLQAVQARLHAISILVYSNALQESANSILYNGLIEELVDVLQITDKQLMEIKAASLRTLTSIVHLERTPKLSSIIDCTGTASYHGFLPVLVRNCIQAMIDPSMDPYPHQFATALFSFLYHLASYDAGGEALVSCGMMEALLKVIKFLGDEQDQITFVTRAVRVVDLITNLDMAAFQSHSGLSIFIYRLEHEVDLCRKECPFVIKPKIQRPNTTQEGEEMETDMDGVQCIPQRAALLKSMLNFLKKAIQDPAFSDGIRHVMDGSLPTSLKHIISNAEYYGPSLFLLATEVVTVFVFQEPSLLSSLQDNGLTDVMLHALLIKDVPATREVLGSLPNVFSALCLNARGLQSFVQCQPFERLFKVLLSPDYLPAMRRRRSSDPLGDTASNLGSAVDELMRHQPTLKTDATTAIIKLLEEICNLGRDPKYICQKPSIQKADGTATAPPPRSNHAAEEASSEDEEEEEVQAMQSFNSTQQNETEPNQQVVGTEERIPIPLMDYILNVMKFVESILSNNTTDDHCQEFVNQKGLLPLVTILGLPNLPIDFPTSAACQAVAGVCKSILTLSHEPKVLQEGLLQLDSILSSLEPLHRPIESPGGSVLLRELACAGNVADATLSAQATPLLHALTAAHAYIMMFVHTCRVGQSEIRSISVNQWGSQLGLSVLSKLSQLYCSLVWESTVLLSLCTPNSLPSGCEFGQADMQKLVPKDEKAGTTQGGKRSDGEQDGAAGSMDASTQGLLEGIGLDGDTLAPMETDEPTASDSKGKSKITPAMAARIKQIKPLLSASSRLGRALAELFGLLVKLCVGSPVRQRRSHHAASTTTAPTPAARSTASALTKLLTKGLSWQPPPYTPTPRFRLTFFICSVGFTSPMLFDERKYPYHLMLQKFLCSGGHNALFETFNWALSMGGKVPVSEGLEHSDLPDGTGEFLDAWLMLVEKMVNPTTVLESPHSLPAKLPGGVQNFPQFSALRFLVVTQKAAFTCIKNLWNRKPLKVYGGRMAESMLAILCHILRGEPVIRERLSKEKEGSRGEEDTGQEEGGSRREPQVNQQQLQQLMDMGFTREHAMEALLNTSTMEQATEYLLTHPPPIMGGVVRDLSMSEEDQMMRAIAMSLGQDIPMDQRAESPEEVACRKEEEERKAREKQEEEEAKCLEKFQDADPLEQDELHTFTDTMLPGCFHLLDELPDTVYRVCDLIMTAIKRNGADYRDMILKQVVNQVWEAADVLIKAALPLTTSDTKTVSEWISQMATLPQASNLATRILLLTLLFEELKLPCAWVVESSGILNVLIKLLEVVQPCLQAAKEQKEVQTPKWITPVLLLIDFYEKTAISSKRRAQMTKYLQSNSNNWRWFDDRSGRWCSYSASNNSTIDSAWKSGETSVRFTAGRRRYTVQFTTMVQVNEETGNRRPVMLTLLRVPRLNKNSKNSNGQELEKTLEESKEMDIKRKENKGNDTPLALESTNTEKETSLEETKIGEILIQGLTEDMVTVLIRACVSMLGVPVDPDTLHATLRLCLRLTRDHKYAMMFAELKSTRMILNLTQSSGFNGFTPLVTLLLRHIIEDPCTLRHTMEKVVRSAATSGAGSTTSGVVSGSLGSREINYILRVLGPAACRNPDIFTEVANCCIRIALPAPRGSGTASDDEFENLRIKGPNAVQLVKTTPLKPSPLPVIPDTIKEVIYDMLNALAAYHAPEEADKSDPKPGVMTQEVGQLLQDMGDDVYQQYRSLTRQSSDFDTQSGFSINSQVFAADGASTETSASGTSQGEASTPEESRDGKKDKEGDRASEEGKQKGKGSKPLMPTSTILRLLAELVRSYVGIATLIANYSYTVGQSELIKEDCSVLAFVLDHLLPHTQNAEDKDTPALARLFLASLAAAGSGTDAQVALVNEVKAALGRALAMAESTEKHARLQAVMCIISTIMESCPSTSSFYSSATAKTQHNGMNNIIRLFLKKGLVNDLARVPHSLDLSSPNMANTVNAALKPLETLSRIVNQPSSLFGSKSASSKNKSEQDAQGASQDSSSNQQDPGEPGEAEVQEEDHDVTQTEVADGDIMDGEAETDSVVIAGQPEVLSSQEMQVENELEDLIDELLERDGGSGNSTIIVSRSGEDESQEDVLMDEAPSNLSQASTLQANREDSMNILDPEDEEEHTQEEDSSGSNEDEDDSQDEEEEEEEDEEDDQEDDEGEEGDEDDDDDGSEMELDEDYPDMNASPLVRFERFDREDDLIIEFDNMFSSATDIPPSPGNIPTTHPLMVRHADHSSLTLGSGSSTTRLTQGIGRSQRTLRQLTANTGHTIHVHYPGNRQPNPPLILQRLLGPSAAADILQLSSSLPLQSRGRARLLVGNDDVHIIARSDDELLDDFFHDQSTATSQAGTLSSIPTALTRWTEECKVLDAESMHDCVSVVKVSIVNHLEFLRDEELEERREKRRKQLAEEETKITDKGKEDKENRDQSAQCTASKSNDSTEQNLSDGTPMPDSYPTTPSSTDAATSESKETLGTLQSSQQQPTLPTPPALGEVPQELQSPAGEGGSSTQLLMPVEPEELGPTRPSGEAETTQMELSPAPTITSLSPERAEDSDALTAVSSQLEGSPMDTSSLASCTLEEAVGDTSAAGSSEQPRAGSSTPGDAPPAVAEVQGRSDGSGESAQPPEDSSPPASSESSSTRDSAVAISGADSRGILEEPLPSTSSEEEDPLAGISLPEGVDPSFLAALPDDIRREVLQNQLGIRPPTRTAPSTNSSAPAVVGNPGVTEVSPEFLAALPPAIQEEVLAQQRAEQQRRELAQNASSDTPMDPVTFIQTLPSDLRRSVLEDMEDSVLAVMPPDIAAEAQALRREQEARQRQLMHERLFGHSSTSALSAILRSPAFTSRLSGNRGVQYTRLAVQRGGTFQMGGSSSHNRPSGSNVDTLLRLRGRLLLDHEALSCLLVLLFVDEPKLNTSRLHRVLRNLCYHAQTRHWVIRSLLSILQRSSESELCIETPKLTTSEEKGKKSSKSCGSSSHENRPLDLLHKMESKSSNQLSWLSVSMDAALGCRTNIFQIQRSGGRKHTEKHASGGSTVHIHPQAAPVVCRHVLDTLIQLAKVFPSHFTQQRTKETNCESDRERGNKACSPCSSQSSSSGICTDFWDLLVKLDNMNVSRKGKNSVKSVPVSAGGEGETSPYSLEASPLGQLMNMLSHPVIRRSSLLTEKLLRLLSLISIALPENKVSEAQANSGSGASSTTTATSTTSTTTTTAASTTPTPPTAPTPVTSAPALVAATAISTIVVAASTTVTTPTTATTTVSISPTTKGSKSPAKVSDGGSSSTDFKMVSSGLTENQLQLSVEVLTSHSCSEEGLEDAANVLLQLSRGDSGTRDTVLKLLLNGARHLGYTLCKQIGTLLAELREYNLEQQRRAQCETLSPDGLPEEQPQTTKLKGKMQSRFDMAENVVIVASQKRPLGGRELQLPSMSMLTSKTSTQKFFLRVLQVIIQLRDDTRRANKKAKQTGRLGSSGLGSASSIQAAVRQLEAEADAIIQMVREGQRARRQQQAATSESSQSEASVRREESPMDVDQPSPSAQDTQSIASDGTPQGEKEKEERPPELPLLSEQLSLDELWDMLGECLKELEESHDQHAVLVLQPAVEAFFLVHATERESKPPVRDTRESQLAHIKDEPPPLSPAPLTPATPSSLDPFFSREPSSMHISSSLPPDTQKFLRFAETHRTVLNQILRQSTTHLADGPFAVLVDYIRVLDFDVKRKYFRQELERLDEGLRKEDMAVHVRRDHVFEDSYRELHRKSPEEMKNRLYIVFEGEEGQDAGGLLREWYMIISREMFNPMYALFRTSPGDRVTYTINPSSHCNPNHLSYFKFVGRIVAKAVYDNRLLECYFTRSFYKHILGKSVRYTDMESEDYHFYQGLVYLLENDVSTLGYDLTFSTEVQEFGVCEVRDLKPNGANILVTEENKKEYVHLVCQMRMTGAIRKQLAAFLEGFYEIIPKRLISIFTEQELELLISGLPTIDIDDLKSNTEYHKYQSNSIQIQWFWRALRSFDQADRAKFLQFVTGTSKVPLQGFAALEGMNGIQKFQIHRDDRSTDRLPSAHTCFNQLDLPAYESFEKLRHMLLLAIQECSEGFGLA",
# I2BP1_HUMAN
"MASVQASRRQWCYLCDLPKMPWAMVWDFSEAVCRGCVNFEGADRIELLIDAARQLKRSHVLPEGRSPGPPALKHPATKDLAAAAAQGPQLPPPQAQPQPSGTGGGVSGQDRYDRATSSGRLPLPSPALEYTLGSRLANGLGREEAVAEGARRALLGSMPGLMPPGLLAAAVSGLGSRGLTLAPGLSPARPLFGSDFEKEKQQRNADCLAELNEAMRGRAEEWHGRPKAVREQLLALSACAPFNVRFKKDHGLVGRVFAFDATARPPGYEFELKLFTEYPCGSGNVYAGVLAVARQMFHDALREPGKALASSGFKYLEYERRHGSGEWRQLGELLTDGVRSFREPAPAEALPQQYPEPAPAALCGPPPRAPSRNLAPTPRRRKASPEPEGEAAGKMTTEEQQQRHWVAPGGPYSAETPGVPSPIAALKNVAEALGHSPKDPGGGGGPVRAGGASPAASSTAQPPTQHRLVARNGEAEVSPTAGAEAVSGGGSGTGATPGAPLCCTLCRERLEDTHFVQCPSVPGHKFCFPCSREFIKAQGPAGEVYCPSGDKCPLVGSSVPWAFMQGEIATILAGDIKVKKERDP",
# I2BP2_HUMAN
"MAAAVAVAAASRRQSCYLCDLPRMPWAMIWDFTEPVCRGCVNYEGADRVEFVIETARQLKRAHGCFPEGRSPPGAAASAAAKPPPLSAKDILLQQQQQLGHGGPEAAPRAPQALERYPLAAAAERPPRLGSDFGSSRPAASLAQPPTPQPPPVNGILVPNGFSKLEEPPELNRQSPNPRRGHAVPPTLVPLMNGSATPLPTALGLGGRAAASLAAVSGTAAASLGSAQPTDLGAHKRPASVSSSAAVEHEQREAAAKEKQPPPPAHRGPADSLSTAAGAAELSAEGAGKSRGSGEQDWVNRPKTVRDTLLALHQHGHSGPFESKFKKEPALTAGRLLGFEANGANGSKAVARTARKRKPSPEPEGEVGPPKINGEAQPWLSTSTEGLKIPMTPTSSFVSPPPPTASPHSNRTTPPEAAQNGQSPMAALILVADNAGGSHASKDANQVHSTTRRNSNSPPSPSSMNQRRLGPREVGGQGAGNTGGLEPVHPASLPDSSLATSAPLCCTLCHERLEDTHFVQCPSVPSHKFCFPCSRQSIKQQGASGEVYCPSGEKCPLVGSNVPWAFMQGEIATILAGDVKVKKERDS",
# I2BPL_HUMAN
"MSAAQVSSSRRQSCYLCDLPRMPWAMIWDFSEPVCRGCVNYEGADRIEFVIETARQLKRAHGCFQDGRSPGPPPPVGVKTVALSAKEAAAAAAAAAAAAAAAQQQQQQQQQQQQQQQQQQQQQQQQQLNHVDGSSKPAVLAAPSGLERYGLSAAAAAAAAAAAAVEQRSRFEYPPPPVSLGSSSHTARLPNGLGGPNGFPKPTPEEGPPELNRQSPNSSSAAASVASRRGTHGGLVTGLPNPGGGGGPQLTVPPNLLPQTLLNGPASAAVLPPPPPHALGSRGPPTPAPPGAPGGPACLGGTPGVSATSSSASSSTSSSVAEVGVGAGGKRPGSVSSTDQERELKEKQRNAEALAELSESLRNRAEEWASKPKMVRDTLLTLAGCTPYEVRFKKDHSLLGRVFAFDAVSKPGMDYELKLFIEYPTGSGNVYSSASGVAKQMYQDCMKDFGRGLSSGFKYLEYEKKHGSGDWRLLGDLLPEAVRFFKEGVPGADMLPQPYLDASCPMLPTALVSLSRAPSAPPGTGALPPAAPSGRGAAASLRKRKASPEPPDSAEGALKLGEEQQRQQWMANQSEALKLTMSAGGFAAPGHAAGGPPPPPPPLGPHSNRTTPPESAPQNGPSPMAALMSVADTLGTAHSPKDGSSVHSTTASARRNSSSPVSPASVPGQRRLASRNGDLNLQVAPPPPSAHPGMDQVHPQNIPDSPMANSGPLCCTICHERLEDTHFVQCPSVPSHKFCFPCSRESIKAQGATGEVYCPSGEKCPLVGSNVPWAFMQGEIATILAGDVKVKKERDP",
# ITCH_HUMAN
"MSDSGSQLGSMGSLTMKSQLQITVISAKLKENKKNWFGPSPYVEVTVDGQSKKTEKCNNTNSPKWKQPLTVIVTPVSKLHFRVWSHQTLKSDVLLGTAALDIYETLKSNNMKLEEVVVTLQLGGDKEPTETIGDLSICLDGLQLESEVVTNGETTCSENGVSLCLPRLECNSAISAHCNLCLPGLSDSPISASRVAGFTGASQNDDGSRSKDETRVSTNGSDDPEDAGAGENRRVSGNNSPSLSNGGFKPSRPPRPSRPPPPTPRRPASVNGSPSATSESDGSSTGSLPPTNTNTNTSEGATSGLIIPLTISGGSGPRPLNPVTQAPLPPGWEQRVDQHGRVYYVDHVEKRTTWDRPEPLPPGWERRVDNMGRIYYVDHFTRTTTWQRPTLESVRNYEQWQLQRSQLQGAMQQFNQRFIYGNQDLFATSQSKEFDPLGPLPPGWEKRTDSNGRVYFVNHNTRITQWEDPRSQGQLNEKPLPEGWEMRFTVDGIPYFVDHNRRTTTYIDPRTGKSALDNGPQIAYVRDFKAKVQYFRFWCQQLAMPQHIKITVTRKTLFEDSFQQIMSFSPQDLRRRLWVIFPGEEGLDYGGVAREWFFLLSHEVLNPMYCLFEYAGKDNYCLQINPASYINPDHLKYFRFIGRFIAMALFHGKFIDTGFSLPFYKRILNKPVGLKDLESIDPEFYNSLIWVKENNIEECDLEMYFSVDKEILGEIKSHDLKPNGGNILVTEENKEEYIRMVAEWRLSRGVEEQTQAFFEGFNEILPQQYLQYFDAKELEVLLCGMQEIDLNDWQRHAIYRHYARTSKQIMWFWQFVKEIDNEKRMRLLQFVTGTCRLPVGGFADLMGSNGPQKFCIEKVGKENWLPRSHTCFNRLDLPPYKSYEQLKEKLLFAIEETEGFGQE",
# KCMF1_HUMAN
"MSRHEGVSCDACLKGNFRGRRYKCLICYDYDLCASCYESGATTTRHTTDHPMQCILTRVDFDLYYGGEAFSVEQPQSFTCPYCGKMGYTETSLQEHVTSEHAETSTEVICPICAALPGGDPNHVTDDFAAHLTLEHRAPRDLDESSGVRHVRRMFHPGRGLGGPRARRSNMHFTSSSTGGLSSSQSSYSPSNREAMDPIAELLSQLSGVRRSAGGQLNSSGPSASQLQQLQMQLQLERQHAQAARQQLETARNATRRTNTSSVTTTITQSTATTNIANTESSQQTLQNSQFLLTRLNDPKMSETERQSMESERADRSLFVQELLLSTLVREESSSSDEDDRGEMADFGAMGCVDIMPLDVALENLNLKESNKGNEPPPPPL",
# KMT2C_HUMAN
"MSSEEDKSVEQPQPPPPPPEEPGAPAPSPAAADKRPRGRPRKDGASPFQRARKKPRSRGKTAVEDEDSMDGLETTETETIVETEIKEQSAEEDAEAEVDNSKQLIPTLQRSVSEESANSLVSVGVEAKISEQLCAFCYCGEKSSLGQGDLKQFRITPGFILPWRNQPSNKKDIDDNSNGTYEKMQNSAPRKQRGQRKERSPQQNIVSCVSVSTQTASDDQAGKLWDELSLVGLPDAIDIQALFDSTGTCWAHHRCVEWSLGVCQMEEPLLVNVDKAVVSGSTERCAFCKHLGATIKCCEEKCTQMYHYPCAAGAGTFQDFSHIFLLCPEHIDQAPERSKEDANCAVCDSPGDLLDQFFCTTCGQHYHGMCLDIAVTPLKRAGWQCPECKVCQNCKQSGEDSKMLVCDTCDKGYHTFCLQPVMKSVPTNGWKCKNCRICIECGTRSSSQWHHNCLICDNCYQQQDNLCPFCGKCYHPELQKDMLHCNMCKRWVHLECDKPTDHELDTQLKEEYICMYCKHLGAEMDRLQPGEEVEIAELTTDYNNEMEVEGPEDQMVFSEQAANKDVNGQESTPGIVPDAVQVHTEEQQKSHPSESLDTDSLLIAVSSQHTVNTELEKQISNEVDSEDLKMSSEVKHICGEDQIEDKMEVTENIEVVTHQITVQQEQLQLLEEPETVVSREESRPPKLVMESVTLPLETLVSPHEESISLCPEEQLVIERLQGEKEQKENSELSTGLMDSEMTPTIEGCVKDVSYQGGKSIKLSSETESSFSSSADISKADVSSSPTPSSDLPSHDMLHNYPSALSSSAGNIMPTTYISVTPKIGMGKPAITKRKFSPGRPRSKQGAWSTHNTVSPPSWSPDISEGREIFKPRQLPGSAIWSIKVGRGSGFPGKRRPRGAGLSGRGGRGRSKLKSGIGAVVLPGVSTADISSNKDDEENSMHNTVVLFSSSDKFTLNQDMCVVCGSFGQGAEGRLLACSQCGQCYHPYCVSIKITKVVLSKGWRCLECTVCEACGKATDPGRLLLCDDCDISYHTYCLDPPLQTVPKGGWKCKWCVWCRHCGATSAGLRCEWQNNYTQCAPCASLSSCPVCYRNYREEDLILQCRQCDRWMHAVCQNLNTEEEVENVADIGFDCSMCRPYMPASNVPSSDCCESSLVAQIVTKVKELDPPKTYTQDGVCLTESGMTQLQSLTVTVPRRKRSKPKLKLKIINQNSVAVLQTPPDIQSEHSRDGEMDDSREGELMDCDGKSESSPEREAVDDETKGVEGTDGVKKRKRKPYRPGIGGFMVRQRSRTGQGKTKRSVIRKDSSGSISEQLPCRDDGWSEQLPDTLVDESVSVTESTEKIKKRYRKRKNKLEETFPAYLQEAFFGKDLLDTSRQSKISLDNLSEDGAQLLYKTNMNTGFLDPSLDPLLSSSSAPTKSGTHGPADDPLADISEVLNTDDDILGIISDDLAKSVDHSDIGPVTDDPSSLPQPNVNQSSRPLSEEQLDGILSPELDKMVTDGAILGKLYKIPELGGKDVEDLFTAVLSPANTQPTPLPQPPPPTQLLPIHNQDAFSRMPLMNGLIGSSPHLPHNSLPPGSGLGTFSAIAQSSYPDARDKNSAFNPMASDPNNSWTSSAPTVEGENDTMSNAQRSTLKWEKEEALGEMATVAPVLYTNINFPNLKEEFPDWTTRVKQIAKLWRKASSQERAPYVQKARDNRAALRINKVQMSNDSMKRQQQQDSIDPSSRIDSELFKDPLKQRESEHEQEWKFRQQMRQKSKQQAKIEATQKLEQVKNEQQQQQQQQFGSQHLLVQSGSDTPSSGIQSPLTPQPGNGNMSPAQSFHKELFTKQPPSTPTSTSSDDVFVKPQAPPPPPAPSRIPIQDSLSQAQTSQPPSPQVFSPGSSNSRPPSPMDPYAKMVGTPRPPPVGHSFSRRNSAAPVENCTPLSSVSRPLQMNETTANRPSPVRDLCSSSTTNNDPYAKPPDTPRPVMTDQFPKSLGLSRSPVVSEQTAKGPIAAGTSDHFTKPSPRADVFQRQRIPDSYARPLLTPAPLDSGPGPFKTPMQPPPSSQDPYGSVSQASRRLSVDPYERPALTPRPIDNFSHNQSNDPYSQPPLTPHPAVNESFAHPSRAFSQPGTISRPTSQDPYSQPPGTPRPVVDSYSQSSGTARSNTDPYSQPPGTPRPTTVDPYSQQPQTPRPSTQTDLFVTPVTNQRHSDPYAHPPGTPRPGISVPYSQPPATPRPRISEGFTRSSMTRPVLMPNQDPFLQAAQNRGPALPGPLVRPPDTCSQTPRPPGPGLSDTFSRVSPSAARDPYDQSPMTPRSQSDSFGTSQTAHDVADQPRPGSEGSFCASSNSPMHSQGQQFSGVSQLPGPVPTSGVTDTQNTVNMAQADTEKLRQRQKLREIILQQQQQKKIAGRQEKGSQDSPAVPHPGPLQHWQPENVNQAFTRPPPPYPGNIRSPVAPPLGPRYAVFPKDQRGPYPPDVASMGMRPHGFRFGFPGGSHGTMPSQERFLVPPQQIQGSGVSPQLRRSVSVDMPRPLNNSQMNNPVGLPQHFSPQSLPVQQHNILGQAYIELRHRAPDGRQRLPFSAPPGSVVEASSNLRHGNFIPRPDFPGPRHTDPMRRPPQGLPNQLPVHPDLEQVPPSQQEQGHSVHSSSMVMRTLNHPLGGEFSEAPLSTSVPSETTSDNLQITTQPSDGLEEKLDSDDPSVKELDVKDLEGVEVKDLDDEDLENLNLDTEDGKVVELDTLDNLETNDPNLDDLLRSGEFDIIAYTDPELDMGDKKSMFNEELDLPIDDKLDNQCVSVEPKKKEQENKTLVLSDKHSPQKKSTVTNEVKTEVLSPNSKVESKCETEKNDENKDNVDTPCSQASAHSDLNDGEKTSLHPCDPDLFEKRTNRETAGPSANVIQASTQLPAQDVINSCGITGSTPVLSSLLANEKSDNSDIRPSGSPPPPTLPASPSNHVSSLPPFIAPPGRVLDNAMNSNVTVVSRVNHVFSQGVQVNPGLIPGQSTVNHSLGTGKPATQTGPQTSQSGTSSMSGPQQLMIPQTLAQQNRERPLLLEEQPLLLQDLLDQERQEQQQQRQMQAMIRQRSEPFFPNIDFDAITDPIMKAKMVALKGINKVMAQNNLGMPPMVMSRFPFMGQVVTGTQNSEGQNLGPQAIPQDGSITHQISRPNPPNFGPGFVNDSQRKQYEEWLQETQQLLQMQQKYLEEQIGAHRKSKKALSAKQRTAKKAGREFPEEDAEQLKHVTEQQSMVQKQLEQIRKQQKEHAELIEDYRIKQQQQCAMAPPTMMPSVQPQPPLIPGATPPTMSQPTFPMVPQQLQHQQHTTVISGHTSPVRMPSLPGWQPNSAPAHLPLNPPRIQPPIAQLPIKTCTPAPGTVSNANPQSGPPPRVEFDDNNPFSESFQERERKERLREQQERQRIQLMQEVDRQRALQQRMEMEQHGMVGSEISSSRTSVSQIPFYSSDLPCDFMQPLGPLQQSPQHQQQMGQVLQQQNIQQGSINSPSTQTFMQTNERRQVGPPSFVPDSPSIPVGSPNFSSVKQGHGNLSGTSFQQSPVRPSFTPALPAAPPVANSSLPCGQDSTITHGHSYPGSTQSLIQLYSDIIPEEKGKKKRTRKKKRDDDAESTKAPSTPHSDITAPPTPGISETTSTPAVSTPSELPQQADQESVEPVGPSTPNMAAGQLCTELENKLPNSDFSQATPNQQTYANSEVDKLSMETPAKTEEIKLEKAETESCPGQEEPKLEEQNGSKVEGNAVACPVSSAQSPPHSAGAPAAKGDSGNELLKHLLKNKKSSSLLNQKPEGSICSEDDCTKDNKLVEKQNPAEGLQTLGAQMQGGFGCGNQLPKTDGGSETKKQRSKRTQRTGEKAAPRSKKRKKDEEEKQAMYSSTDTFTHLKQQNNLSNPPTPPASLPPTPPPMACQKMANGFATTEELAGKAGVLVSHEVTKTLGPKPFQLPFRPQDDLLARALAQGPKTVDVPASLPTPPHNNQEELRIQDHCGDRDTPDSFVPSSSPESVVGVEVSRYPDLSLVKEEPPEPVPSPIIPILPSTAGKSSESRRNDIKTEPGTLYFASPFGPSPNGPRSGLISVAITLHPTAAENISSVVAAFSDLLHVRIPNSYEVSSAPDVPSMGLVSSHRINPGLEYRQHLLLRGPPPGSANPPRLVSSYRLKQPNVPFPPTSNGLSGYKDSSHGIAESAALRPQWCCHCKVVILGSGVRKSFKDLTLLNKDSRESTKRVEKDIVFCSNNCFILYSSTAQAKNSENKESIPSLPQSPMRETPSKAFHQYSNNISTLDVHCLPQLPEKASPPASPPIAFPPAFEAAQVEAKPDELKVTVKLKPRLRAVHGGFEDCRPLNKKWRGMKWKKWSIHIVIPKGTFKPPCEDEIDEFLKKLGTSLKPDPVPKDYRKCCFCHEEGDGLTDGPARLLNLDLDLWVHLNCALWSTEVYETQAGALINVELALRRGLQMKCVFCHKTGATSGCHRFRCTNIYHFTCAIKAQCMFFKDKTMLCPMHKPKGIHEQELSYFAVFRRVYVQRDEVRQIASIVQRGERDHTFRVGSLIFHTIGQLLPQQMQAFHSPKALFPVGYEASRLYWSTRYANRRCRYLCSIEEKDGRPVFVIRIVEQGHEDLVLSDISPKGVWDKILEPVACVRKKSEMLQLFPAYLKGEDLFGLTVSAVARIAESLPGVEACENYTFRYGRNPLMELPLAVNPTGCARSEPKMSAHVKRFVLRPHTLNSTSTSKSFQSTVTGELNAPYSKQFVHSKSSQYRKMKTEWKSNVYLARSRIQGLGLYAARDIEKHTMVIEYIGTIIRNEVANRKEKLYESQNRGVYMFRMDNDHVIDATLTGGPARYINHSCAPNCVAEVVTFERGHKIIISSSRRIQKGEELCYDYKFDFEDDQHKIPCHCGAVNCRKWMN",
# KMT2D_HUMAN
"MDSQKLAGEDKDSEPAADGPAASEDPSATESDLPNPHVGEVSVLSSGSPRLQETPQDCSGGPVRRCALCNCGEPSLHGQRELRRFELPFDWPRCPVVSPGGSPGPNEAVLPSEDLSQIGFPEGLTPAHLGEPGGSCWAHHWCAAWSAGVWGQEGPELCGVDKAIFSGISQRCSHCTRLGASIPCRSPGCPRLYHFPCATASGSFLSMKTLQLLCPEHSEGAAYLEEARCAVCEGPGELCDLFFCTSCGHHYHGACLDTALTARKRAGWQCPECKVCQACRKPGNDSKMLVCETCDKGYHTFCLKPPMEELPAHSWKCKACRVCRACGAGSAELNPNSEWFENYSLCHRCHKAQGGQTIRSVAEQHTPVCSRFSPPEPGDTPTDEPDALYVACQGQPKGGHVTSMQPKEPGPLQCEAKPLGKAGVQLEPQLEAPLNEEMPLLPPPEESPLSPPPEESPTSPPPEASRLSPPPEELPASPLPEALHLSRPLEESPLSPPPEESPLSPPPESSPFSPLEESPLSPPEESPPSPALETPLSPPPEASPLSPPFEESPLSPPPEELPTSPPPEASRLSPPPEESPMSPPPEESPMSPPPEASRLFPPFEESPLSPPPEESPLSPPPEASRLSPPPEDSPMSPPPEESPMSPPPEVSRLSPLPVVSRLSPPPEESPLSPPPEESPTSPPPEASRLSPPPEDSPTSPPPEDSPASPPPEDSLMSLPLEESPLLPLPEEPQLCPRSEGPHLSPRPEEPHLSPRPEEPHLSPQAEEPHLSPQPEEPCLCAVPEEPHLSPQAEGPHLSPQPEELHLSPQTEEPHLSPVPEEPCLSPQPEESHLSPQSEEPCLSPRPEESHLSPELEKPPLSPRPEKPPEEPGQCPAPEELPLFPPPGEPSLSPLLGEPALSEPGEPPLSPLPEELPLSPSGEPSLSPQLMPPDPLPPPLSPIITAAAPPALSPLGELEYPFGAKGDSDPESPLAAPILETPISPPPEANCTDPEPVPPMILPPSPGSPVGPASPILMEPLPPQCSPLLQHSLVPQNSPPSQCSPPALPLSVPSPLSPIGKVVGVSDEAELHEMETEKVSEPECPALEPSATSPLPSPMGDLSCPAPSPAPALDDFSGLGEDTAPLDGIDAPGSQPEPGQTPGSLASELKGSPVLLDPEELAPVTPMEVYPECKQTAGQGSPCEEQEEPRAPVAPTPPTLIKSDIVNEISNLSQGDASASFPGSEPLLGSPDPEGGGSLSMELGVSTDVSPARDEGSLRLCTDSLPETDDSLLCDAGTAISGGKAEGEKGRRRSSPARSRIKQGRSSSFPGRRRPRGGAHGGRGRGRARLKSTASSIETLVVADIDSSPSKEEEEEDDDTMQNTVVLFSNTDKFVLMQDMCVVCGSFGRGAEGHLLACSQCSQCYHPYCVNSKITKVMLLKGWRCVECIVCEVCGQASDPSRLLLCDDCDISYHTYCLDPPLLTVPKGGWKCKWCVSCMQCGAASPGFHCEWQNSYTHCGPCASLVTCPICHAPYVEEDLLIQCRHCERWMHAGCESLFTEDDVEQAADEGFDCVSCQPYVVKPVAPVAPPELVPMKVKEPEPQYFRFEGVWLTETGMALLRNLTMSPLHKRRQRRGRLGLPGEAGLEGSEPSDALGPDDKKDGDLDTDELLKGEGGVEHMECEIKLEGPVSPDVEPGKEETEESKKRKRKPYRPGIGGFMVRQRKSHTRTKKGPAAQAEVLSGDGQPDEVIPADLPAEGAVEQSLAEGDEKKKQQRRGRKKSKLEDMFPAYLQEAFFGKELLDLSRKALFAVGVGRPSFGLGTPKAKGDGGSERKELPTSQKGDDGPDIADEESRGLEGKADTPGPEDGGVKASPVPSDPEKPGTPGEGMLSSDLDRISTEELPKMESKDLQQLFKDVLGSEREQHLGCGTPGLEGSRTPLQRPFLQGGLPLGNLPSSSPMDSYPGLCQSPFLDSRERGGFFSPEPGEPDSPWTGSGGTTPSTPTTPTTEGEGDGLSYNQRSLQRWEKDEELGQLSTISPVLYANINFPNLKQDYPDWSSRCKQIMKLWRKVPAADKAPYLQKAKDNRAAHRINKVQKQAESQINKQTKVGDIARKTDRPALHLRIPPQPGALGSPPPAAAPTIFIGSPTTPAGLSTSADGFLKPPAGSVPGPDSPGELFLKLPPQVPAQVPSQDPFGLAPAYPLEPRFPTAPPTYPPYPSPTGAPAQPPMLGASSRPGAGQPGEFHTTPPGTPRHQPSTPDPFLKPRCPSLDNLAVPESPGVGGGKASEPLLSPPPFGESRKALEVKKEELGASSPSYGPPNLGFVDSPSSGTHLGGLELKTPDVFKAPLTPRASQVEPQSPGLGLRPQEPPPAQALAPSPPSHPDIFRPGSYTDPYAQPPLTPRPQPPPPESCCALPPRSLPSDPFSRVPASPQSQSSSQSPLTPRPLSAEAFCPSPVTPRFQSPDPYSRPPSRPQSRDPFAPLHKPPRPQPPEVAFKAGSLAHTSLGAGGFPAALPAGPAGELHAKVPSGQPPNFVRSPGTGAFVGTPSPMRFTFPQAVGEPSLKPPVPQPGLPPPHGINSHFGPGPTLGKPQSTNYTVATGNFHPSGSPLGPSSGSTGESYGLSPLRPPSVLPPPAPDGSLPYLSHGASQRSGITSPVEKREDPGTGMGSSLATAELPGTQDPGMSGLSQTELEKQRQRQRLRELLIRQQIQRNTLRQEKETAAAAAGAVGPPGSWGAEPSSPAFEQLSRGQTPFAGTQDKSSLVGLPPSKLSGPILGPGSFPSDDRLSRPPPPATPSSMDVNSRQLVGGSQAFYQRAPYPGSLPLQQQQQQLWQQQQATAATSMRFAMSARFPSTPGPELGRQALGSPLAGISTRLPGPGEPVPGPAGPAQFIELRHNVQKGLGPGGTPFPGQGPPQRPRFYPVSEDPHRLAPEGLRGLAVSGLPPQKPSAPPAPELNNSLHPTPHTKGPTLPTGLELVNRPPSSTELGRPNPLALEAGKLPCEDPELDDDFDAHKALEDDEELAHLGLGVDVAKGDDELGTLENLETNDPHLDDLLNGDEFDLLAYTDPELDTGDKKDIFNEHLRLVESANEKAEREALLRGVEPGPLGPEERPPPAADASEPRLASVLPEVKPKVEEGGRHPSPCQFTIATPKVEPAPAANSLGLGLKPGQSMMGSRDTRMGTGPFSSSGHTAEKASFGATGGPPAHLLTPSPLSGPGGSSLLEKFELESGALTLPGGPAASGDELDKMESSLVASELPLLIEDLLEHEKKELQKKQQLSAQLQPAQQQQQQQQQHSLLSAPGPAQAMSLPHEGSSPSLAGSQQQLSLGLAGARQPGLPQPLMPTQPPAHALQQRLAPSMAMVSNQGHMLSGQHGGQAGLVPQQSSQPVLSQKPMGTMPPSMCMKPQQLAMQQQLANSFFPDTDLDKFAAEDIIDPIAKAKMVALKGIKKVMAQGSIGVAPGMNRQQVSLLAQRLSGGPSSDLQNHVAAGSGQERSAGDPSQPRPNPPTFAQGVINEADQRQYEEWLFHTQQLLQMQLKVLEEQIGVHRKSRKALCAKQRTAKKAGREFPEADAEKLKLVTEQQSKIQKQLDQVRKQQKEHTNLMAEYRNKQQQQQQQQQQQQQQHSAVLALSPSQSPRLLTKLPGQLLPGHGLQPPQGPPGGQAGGLRLTPGGMALPGQPGGPFLNTALAQQQQQQHSGGAGSLAGPSGGFFPGNLALRSLGPDSRLLQERQLQLQQQRMQLAQKLQQQQQQQQQQQHLLGQVAIQQQQQQGPGVQTNQALGPKPQGLMPPSSHQGLLVQQLSPQPPQGPQGMLGPAQVAVLQQQHPGALGPQGPHRQVLMTQSRVLSSPQLAQQGQGLMGHRLVTAQQQQQQQQHQQQGSMAGLSHLQQSLMSHSGQPKLSAQPMGSLQQLQQQQQLQQQQQLQQQQQQQLQQQQQLQQQQLQQQQQQQQLQQQQQQQLQQQQQQLQQQQQQQQQQFQQQQQQQQMGLLNQSRTLLSPQQQQQQQVALGPGMPAKPLQHFSSPGALGPTLLLTGKEQNTVDPAVSSEATEGPSTHQGGPLAIGTTPESMATEPGEVKPSLSGDSQLLLVQPQPQPQPSSLQLQPPLRLPGQQQQQVSLLHTAGGGSHGQLGSGSSSEASSVPHLLAQPSVSLGDQPGSMTQNLLGPQQPMLERPMQNNTGPQPPKPGPVLQSGQGLPGVGIMPTVGQLRAQLQGVLAKNPQLRHLSPQQQQQLQALLMQRQLQQSQAVRQTPPYQEPGTQTSPLQGLLGCQPQLGGFPGPQTGPLQELGAGPRPQGPPRLPAPPGALSTGPVLGPVHPTPPPSSPQEPKRPSQLPSPSSQLPTEAQLPPTHPGTPKPQGPTLEPPPGRVSPAAAQLADTLFSKGLGPWDPPDNLAETQKPEQSSLVPGHLDQVNGQVVPEASQLSIKQEPREEPCALGAQSVKREANGEPIGAPGTSNHLLLAGPRSEAGHLLLQKLLRAKNVQLSTGRGSEGLRAEINGHIDSKLAGLEQKLQGTPSNKEDAAARKPLTPKPKRVQKASDRLVSSRKKLRKEDGVRASEALLKQLKQELSLLPLTEPAITANFSLFAPFGSGCPVNGQSQLRGAFGSGALPTGPDYYSQLLTKNNLSNPPTPPSSLPPTPPPSVQQKMVNGVTPSEELGEHPKDAASARDSERALRDTSEVKSLDLLAALPTPPHNQTEDVRMESDEDSDSPDSIVPASSPESILGEEAPRFPHLGSGRWEQEDRALSPVIPLIPRASIPVFPDTKPYGALGLEVPGKLPVTTWEKGKGSEVSVMLTVSAAAAKNLNGVMVAVAELLSMKIPNSYEVLFPESPARAGTEPKKGEAEGPGGKEKGLEGKSPDTGPDWLKQFDAVLPGYTLKSQLDILSLLKQESPAPEPPTQHSYTYNVSNLDVRQLSAPPPEEPSPPPSPLAPSPASPPTEPLVELPTEPLAEPPVPSPLPLASSPESARPKPRARPPEEGEDSRPPRLKKWKGVRWKRLRLLLTIQKGSGRQEDEREVAEFMEQLGTALRPDKVPRDMRRCCFCHEEGDGATDGPARLLNLDLDLWVHLNCALWSTEVYETQGGALMNVEVALHRGLLTKCSLCQRTGATSSCNRMRCPNVYHFACAIRAKCMFFKDKTMLCPMHKIKGPCEQELSSFAVFRRVYIERDEVKQIASIIQRGERLHMFRVGGLVFHAIGQLLPHQMADFHSATALYPVGYEATRIYWSLRTNNRRCCYRCSIGENNGRPEFVIKVIEQGLEDLVFTDASPQAVWNRIIEPVAAMRKEADMLRLFPEYLKGEELFGLTVHAVLRIAESLPGVESCQNYLFRYGRHPLMELPLMINPTGCARSEPKILTHYKRPHTLNSTSMSKAYQSTFTGETNTPYSKQFVHSKSSQYRRLRTEWKNNVYLARSRIQGLGLYAAKDLEKHTMVIEYIGTIIRNEVANRREKIYEEQNRGIYMFRINNEHVIDATLTGGPARYINHSCAPNCVAEVVTFDKEDKIIIISSRRIPKGEELTYDYQFDFEDDQHKIPCHCGAWNCRKWMN",
# LNX1_HUMAN
"MNQPESANDPEPLCAVCGQAHSLEENHFYSYPEEVDDDLICHICLQALLDPLDTPCGHTYCTLCLTNFLVEKDFCPMDRKPLVLQHCKKSSILVNKLLNKLLVTCPFREHCTQVLQRCDLEHHFQTSCKGASHYGLTKDRKRRSQDGCPDGCASLTATAPSPEVSAAATISLMTDEPGLDNPAYVSSAEDGQPAISPVDSGRSNRTRARPFERSTIRSRSFKKINRALSVLRRTKSGSAVANHADQGRENSENTTAPEVFPRLYHLIPDGEITSIKINRVDPSESLSIRLVGGSETPLVHIIIQHIYRDGVIARDGRLLPGDIILKVNGMDISNVPHNYAVRLLRQPCQVLWLTVMREQKFRSRNNGQAPDAYRPRDDSFHVILNKSSPEEQLGIKLVRKVDEPGVFIFNVLDGGVAYRHGQLEENDRVLAINGHDLRYGSPESAAHLIQASERRVHLVVSRQVRQRSPDIFQEAGWNSNGSWSPGPGERSNTPKPLHPTITCHEKVVNIQKDPGESLGMTVAGGASHREWDLPIYVISVEPGGVISRDGRIKTGDILLNVDGVELTEVSRSEAVALLKRTSSSIVLKALEVKEYEPQEDCSSPAALDSNHNMAPPSDWSPSWVMWLELPRCLYNCKDIVLRRNTAGSLGFCIVGGYEEYNGNKPFFIKSIVEGTPAYNDGRIRCGDILLAVNGRSTSGMIHACLARLLKELKGRITLTIVSWPGTFL",
# LNX2_HUMAN
"MGTTSDEMVSVEQTSSSSLNPLCFECGQQHWTRENHLYNYQNEVDDDLVCHICLQPLLQPLDTPCGHTFCYKCLRNFLQEKDFCPLDRKRLHFKLCKKSSILVHKLLDKLLVLCPFSSVCKDVMQRCDLEAHLKNRCPGASHRRVALERRKTSRTQAEIENENGPTLLDPAGTLSPEADCLGTGAVPVERHLTSASLSTWSEEPGLDNPAFEESAGADTTQQPLSLPEGEITTIEIHRSNPYIQLGISIVGGNETPLINIVIQEVYRDGVIARDGRLLAGDQILQVNNYNISNVSHNYARAVLSQPCNTLHLTVLRERRFGNRAHNHSDSNSPREEIFQVALHKRDSGEQLGIKLVRRTDEPGVFILDLLEGGLAAQDGRLSSNDRVLAINGHDLKYGTPELAAQIIQASGERVNLTIARPGKPQPGNTIREAGNHSSSSQHHTPPPYYSRPSSHKDLTQCVTCQEKHITVKKEPHESLGMTVAGGRGSKSGELPIFVTSVPPHGCLARDGRIKRGDVLLNINGIDLTNLSHSEAVAMLKASAASPAVALKALEVQIVEEATQNAEEQPSTFSENEYDASWSPSWVMWLGLPSTLHSCHDIVLRRSYLGSWGFSIVGGYEENHTNQPFFIKTIVLGTPAYYDGRLKCGDMIVAVNGLSTVGMSHSALVPMLKEQRNKVTLTVICWPGSLV",
# LONF1_HUMAN
"MSSPAVARTSPGGSREMAPAPQGRGRFWEVGGGSGHRLERAAAESERWELLLRRGELLALGGHLKGALEAFAAALRRGAPARPECLGALVDCLVFNYRLRHGLGWSAAPVAGADGGAGGLLRCLGCRGFLSEPVTVPCGHSYCRRCLRRELRARCRLCRDRLPPATASATDAEGTAPRPPPLAAAIAASDFRTSVVLNHLAEKWFPGQRERARAAGRLGELLHQGRYREALAAACEALRAEPSDLIVKIYRAESYAGLQEFKAAIEDLNAVLFQLPDWPEVYFRKGKVLCDAGFLGDALQLFLQCLALDEDFAPAKLQVQKILCDLLLPENLKEGLKESSWSSLPCTKNRPFDFHSVMEESQSLNEPSPKQSEEIPEVTSEPVKGSLNRAQSAQSINSTEMPAREDCLKRVSSEPVLSVQEKGVLLKRKLSLLEQDVIVNEDGRNKLKKQGETPNEVCMFSLAYGDIPEELIDVSDFECSLCMRLFFEPVTTPCGHSFCKNCLERCLDHAPYCPLCKESLKEYLADRRYCVTQLLEELIVKYLPDELSERKKIYDEETAELSHLTKNVPIFVCTMAYPTVPCPLHVFEPRYRLMIRRSIQTGTKQFGMCVSDTQNSFADYGCMLQIRNVHFLPDGRSVVDTVGGKRFRVLKRGMKDGYCTADIEYLEDVKVENEDEIKNLRELHDLVYSQACSWFQNLRDRFRSQILQHFGSMPEREENLQAAPNGPAWCWWLLAVLPVDPRYQLSVLSMKSLKERLTKIQHILTYFSRDQSK",
# LONF2_HUMAN
"MSPEPVPPPPPPQCPGCDRAEPIAQRLEEGDEAFRAGDYEMAAELFRSMLAGLAQPDRGLCLRLGDALARAGRLPEALGAFRGAARLGALRPEELEELAGGLVRAVGLRDRPLSAENPGGEPEAPGEGGPAPEPRAPRDLLGCPRCRRLLHKPVTLPCGLTVCKRCVEPGPARPQVRRVNVVLSGLLEKCFPAECRLRRLAGQARSLQRQQQPEAALLRCDQALELAPDDNSLLLLRAELYLTMKNYEQALQDASAACQNEPLLIKGHQVKAQALSGLGRSKEVLKEFLYCLALNPECNSVKKEAQKVMCEVLFSATANVHENLTSSIQSRLKAQGHSHMNAQALLEEGDAGSSENSSEKSDMLGNTNSSVLYFILGLHFEEDKKALESILPTAPSAGLKRQFPDDVEDAPDLNAPGKIPKKDLSLQRSPNSETEESQGLSLDVTDFECALCMRLLFEPVTTPCGHTFCLKCLERCLDHAPHCPLCKDKLSELLASRNFNITVLAEELIFRYLPDELSDRKRIYDEEMSELSNLTRDVPIFVCAMAFPTVPCPLHVFEPRYRLMIRRCMETGTKRFGMCLSAEHAGLSEYGCMLEIKDVRTFPDGSSVVDAIGISRFRVLSHRHRDGYNTADIEYLEDEKVEGPEYEELAALHDSVHQQSVSWFASLQDRMKEQILSHFGVMPDREPEPQSNPSGPAWSWWILAVLPLERKAQLAILGMTSLKERLLAIRRILVIITRKMNSRQELANARERNN",
# LONF3_HUMAN
"MESVRIEQMLSLPAEVSSDNLESAERGASAAQVDMGPHPKVAAEGPAPLPTREPEQEQSPGTSTPESKVLLTQADALASRGRIREALEVYRQLSERQQLVAEQLEQLVRCLAEKVPQGEALAPAPPDEGSTASGTVAAEETGAAAAAAATEVWDGFKCRKCHGFLSDPVSLSCGHTFCKLCLERGRAADRRCALCGVKLSALMVATGRARGARRAGQQPPPPLRVNVVLSGLLGKLFPGPARASQLRHEGNRLYRERQVEAALLKYNEAVKLAPNDHLLYSNRSQIYFTLESHENALHDAEIACKLRPMGFKAHFRKAQALATLGKVEEALREFLYCVSLDGKNKRARCEAQRDNLELPHCSSQEEAAARGDGSSLMDPAKVKGDGQQHHMKDQEEEEEKWDATSPKAASSKTGKCQEKKRKHCQIESQEETGMPNKASKQDPPTDQGDKPALSLPLASFDASDLECALCMRLFYEPVTTPCGHTFCLKCLERCLDHNAKCPLCKDGLSQCLASRKYSKNVIMEELIAKFLPEELKERRKLYEEEMEELSNLNKNVPIFVCTMAYPTVPCPLHIFEPCYRLMIRRCIETGTRQFGMCLGDPVKGFAEYGCILEIRNVQFFADGRSVVDSIGKRRFRVLHQSQRDGYNTADIEYIEDQKVQGEDCAELMGLHNCVYQQASLWFHSLKLSLKNRILNHFGPMPEKDADPQMNPNGPAWCWWMLAVLPLESRAQLPFLAMRSLKDRLNGIRRVLAFISRNQN",
# LRSM1_HUMAN
"MPLFFRKRKPSEEARKRLEYQMCLAKEAGADDILDISKCELSEIPFGAFATCKVLQKKVLIVHTNHLTSLLPKSCSLLSLATIKVLDLHDNQLTALPDDLGQLTALQVLNVERNQLMQLPRSIGNLTQLQTLNVKDNKLKELPDTVGELRSLRTLNISGNEIQRLPQMLAHVRTLEMLSLDASAMVYPPREVCGAGTAAILQFLCKESGLEYYPPSQYLLPILEQDGIENSRDSPDGPTDRFSREELEWQNRFSDYEKRKEQKMLEKLEFERRLELGQREHTQLLQQSSSQKDEILQTVKEEQSRLEQGLSEHQRHLNAERQRLQEQLKQTEQNISSRIQKLLQDNQRQKKSSEILKSLENERIRMEQLMSITQEETESLRRRDVASAMQQMLTESCKNRLIQMAYESQRQNLVQQACSSMAEMDERFQQILSWQQMDQNKAISQILQESAMQKAAFEALQVKKDLMHRQIRSQIKLIETELLQLTQLELKRKSLDTESLQEMISEQRWALSSLLQQLLKEKQQREEELREILTELEAKSETRQENYWLIQYQRLLNQKPLSLKLQEEGMERQLVALLEELSAEHYLPIFAHHRLSLDLLSQMSPGDLAKVGVSEAGLQHEILRRVQELLDAARIQPELKPPMGEVVTPTAPQEPPESVRPSAPPAELEVQASECVVCLEREAQMIFLNCGHVCCCQQCCQPLRTCPLCRQDIAQRLRIYHSS",
# LTN1_HUMAN
"MGGKNKQRTKGNLRPSNSGRAAELLAKEQGTVPGFIGFGTSQSDLGYVPAIQGAEEIDSLVDSDFRMVLRKLSKKDVTTKLKAMQEFGTMCTERDTETVKGVLPYWPRIFCKISLDHDRRVREATQQAFEKLILKVKKQLAPYLKSLMGYWLMAQCDTYTPAAFAAKDAFEAAFPPSKQPEAIAFCKDEITSVLQDHLIKETPDTLSDPQTVPEEEREAKFYRVVTCSLLALKRLLCLLPDNELDSLEEKFKSLLSQNKFWKYGKHSVPQIRSAYFELVSALCQRIPQLMKEEASKVSPSVLLSIDDSDPIVCPALWEAVLYTLTTIEDCWLHVNAKKSVFPKLSTVIREGGRGLATVIYPYLLPFISKLPQSITNPKLDFFKNFLTSLVAGLSTERTKTSSLESSAVISAFFECLRFIMQQNLGEEEIEQMLVNDQLIPFIDAVLKDPGLQHGQLFNHLAETLSSWEAKADTEKDEKTAHNLENVLIHFWERLSEICVAKISEPEADVESVLGVSNLLQVLQKPKSSLKSSKKKNGKVRFADEILESNKENEKCVSSEGEKIEGWELTTEPSLTHNSSGLLSPLRKKPLEDLVCKLADISINYVNERKSEQHLRFLSTLLDSFSSSRVFKMLLGDEKQSIVQAKPLEIAKLVQKNPAVQFLYQKLIGWLNEDQRKDFGFLVDILYSALRCCDNDMERKKVLDDLTKVDLKWNSLLKIIEKACPSSDKHALVTPWLKGDILGEKLVNLADCLCNEDLESRVSSESHFSERWTLLSLVLSQHVKNDYLIGDVYVERIIVRLHETLFKTKKLSEAESSDSSVSFICDVAYNYFSSAKGCLLMPSSEDLLLTLFQLCAQSKEKTHLPDFLICKLKNTWLSGVNLLVHQTDSSYKESTFLHLSALWLKNQVQASSLDINSLQVLLSAVDDLLNTLLESEDSYLMGVYIGSVMPNDSEWEKMRQSLPMQWLHRPLLEGRLSLNYECFKTDFKEQDIKTLPSHLCTSALLSKMVLIALRKETVLENNELEKIIAELLYSLQWCEELDNPPIFLIGFCEILQKMNITYDNLRVLGNTSGLLQLLFNRSREHGTLWSLIIAKLILSRSISSDEVKPHYKRKESFFPLTEGNLHTIQSLCPFLSKEEKKEFSAQCIPALLGWTKKDLCSTNGGFGHLAIFNSCLQTKSIDDGELLHGILKIIISWKKEHEDIFLFSCNLSEASPEVLGVNIEIIRFLSLFLKYCSSPLAESEWDFIMCSMLAWLETTSENQALYSIPLVQLFACVSCDLACDLSAFFDSTTLDTIGNLPVNLISEWKEFFSQGIHSLLLPILVTVTGENKDVSETSFQNAMLKPMCETLTYISKEQLLSHKLPARLVADQKTNLPEYLQTLLNTLAPLLLFRARPVQIAVYHMLYKLMPELPQYDQDNLKSYGDEEEEPALSPPAALMSLLSIQEDLLENVLGCIPVGQIVTIKPLSEDFCYVLGYLLTWKLILTFFKAASSQLRALYSMYLRKTKSLNKLLYHLFRLMPENPTYAETAVEVPNKDPKTFFTEELQLSIRETTMLPYHIPHLACSVYHMTLKDLPAMVRLWWNSSEKRVFNIVDRFTSKYVSSVLSFQEISSVQTSTQLFNGMTVKARATTREVMATYTIEDIVIELIIQLPSNYPLGSIIVESGKRVGVAVQQWRNWMLQLSTYLTHQNGSIMEGLALWKNNVDKRFEGVEDCMICFSVIHGFNYSLPKKACRTCKKKFHSACLYKWFTSSNKSTCPLCRETFF",
# MAEA_HUMAN
"MAVQESAAQLSMTLKVQEYPTLKVPYETLNKRFRAAQKNIDRETSHVTMVVAELEKTLSGCPAVDSVVSLLDGVVEKLSVLKRKAVESIQAEDESAKLCKRRIEHLKEHSSDQPAAASVWKRKRMDRMMVEHLLRCGYYNTAVKLARQSGIEDLVNIEMFLTAKEVEESLERRETATCLAWCHDNKSRLRKMKSCLEFSLRIQEFIELIRQNKRLDAVRHARKHFSQAEGSQLDEVRQAMGMLAFPPDTHISPYKDLLDPARWRMLIQQFRYDNYRLHQLGNNSVFTLTLQAGLSAIKTPQCYKEDGSSKSPDCPVCSRSLNKLAQPLPMAHCANSRLVCKISGDVMNENNPPMMLPNGYVYGYNSLLSIRQDDKVVCPRTKEVFHFSQAEKVYIM",
# M3K1_HUMAN
"MAAAAGNRASSSGFPGARATSPEAGGGGGALKASSAPAAAAGLLREAGSGGRERADWRRRQLRKVRSVELDQLPEQPLFLAASPPASSTSPSPEPADAAGSGTGFQPVAVPPPHGAASRGGAHLTESVAAPDSGASSPAAAEPGEKRAPAAEPSPAAAPAGREMENKETLKGLHKMDDRPEERMIREKLKATCMPAWKHEWLERRNRRGPVVVKPIPVKGDGSEMNHLAAESPGEVQASAASPASKGRRSPSPGNSPSGRTVKSESPGVRRKRVSPVPFQSGRITPPRRAPSPDGFSPYSPEETNRRVNKVMRARLYLLQQIGPNSFLIGGDSPDNKYRVFIGPQNCSCARGTFCIHLLFVMLRVFQLEPSDPMLWRKTLKNFEVESLFQKYHSRRSSRIKAPSRNTIQKFVSRMSNSHTLSSSSTSTSSSENSIKDEEEQMCPICLLGMLDEESLTVCEDGCRNKLHHHCMSIWAEECRRNREPLICPLCRSKWRSHDFYSHELSSPVDSPSSLRAAQQQTVQQQPLAGSRRNQESNFNLTHYGTQQIPPAYKDLAEPWIQVFGMELVGCLFSRNWNVREMALRRLSHDVSGALLLANGESTGNSGGSSGSSPSGGATSGSSQTSISGDVVEACCSVLSMVCADPVYKVYVAALKTLRAMLVYTPCHSLAERIKLQRLLQPVVDTILVKCADANSRTSQLSISTLLELCKGQAGELAVGREILKAGSIGIGGVDYVLNCILGNQTESNNWQELLGRLCLIDRLLLEFPAEFYPHIVSTDVSQAEPVEIRYKKLLSLLTFALQSIDNSHSMVGKLSRRIYLSSARMVTTVPHVFSKLLEMLSVSSSTHFTRMRRRLMAIADEVEIAEAIQLGVEDTLDGQQDSFLQASVPNNYLETTENSSPECTVHLEKTGKGLCATKLSASSEDISERLASISVGPSSSTTTTTTTTEQPKPMVQTKGRPHSQCLNSSPLSHHSQLMFPALSTPSSSTPSVPAGTATDVSKHRLQGFIPCRIPSASPQTQRKFSLQFHRNCPENKDSDKLSPVFTQSRPLPSSNIHRPKPSRPTPGNTSKQGDPSKNSMTLDLNSSSKCDDSFGCSSNSSNAVIPSDETVFTPVEEKCRLDVNTELNSSIEDLLEASMPSSDTTVTFKSEVAVLSPEKAENDDTYKDDVNHNQKCKEKMEAEEEEALAIAMAMSASQDALPIVPQLQVENGEDIIIIQQDTPETLPGHTKAKQPYREDTEWLKGQQIGLGAFSSCYQAQDVGTGTLMAVKQVTYVRNTSSEQEEVVEALREEIRMMSHLNHPNIIRMLGATCEKSNYNLFIEWMAGGSVAHLLSKYGAFKESVVINYTEQLLRGLSYLHENQIIHRDVKGANLLIDSTGQRLRIADFGAAARLASKGTGAGEFQGQLLGTIAFMAPEVLRGQQYGRSCDVWSVGCAIIEMACAKPPWNAEKHSNHLALIFKIASATTAPSIPSHLSPGLRDVALRCLELQPQDRPPSRELLKHPVFRTTW",
# MARH1_HUMAN
"MLGWCEAIARNPHRIPNNTRTPEISGDLADASQTSTLNEKSPGRSASRSSNISKASSPTTGTAPRSQSRLSVCPSTQDICRICHCEGDEESPLITPCRCTGTLRFVHQSCLHQWIKSSDTRCCELCKYDFIMETKLKPLRKWEKLQMTTSERRKIFCSVTFHVIAITCVVWSLYVLIDRTAEEIKQGNDNGVLEWPFWTKLVVVAIGFTGGLVFMYVQCKVYVQLWRRLKAYNRVIFVQNCPDTAKKLEKNFSCNVNTDIKDAVVVPVPQTGANSLPSAEGGPPEVVSV",
# MARHA_HUMAN
"MLHDARDRQKFFSDVQYLRDMQHKVDSEYQACLRRQEYRRDPNEKKRDQFWGQETSFERSRFSSRSSSKQSSSEEDALTEPRSSIKISAFKCDSKLPAIDQTSVKQKHKSTMTVRKAEKVDPSEPSPADQAPMVLLRKRKPNLRRFTVSPESHSPRASGDRSRQKQQWPAKVPVPRGADQVVQQEGLMCNTKLKRPNQERRNLVPSSQPMTENAPDRAKKGDPSAPSQSELHPALSQAFQGKNSPQVLSEFSGPPLTPTTVGGPRKASFRFRDEDFYSILSLNSRRESDDTEEETQSEECLWVGVRSPCSPSHHKRSRFGGTSTPQAKNKNFEENAENCRGHSSRRSEPSHGSLRISNAMEPATERPSAGQRLSQDPGLPDRESATEKDRGGSENAKKSPLSWDTKSEPRQEVGVNAENVWSDCISVEHRPGTHDSEGYWKDYLNSSQNSLDYFISGRPISPRSSVNSSYNPPASFMHSALRDDIPVDLSMSSTSVHSSDSEGNSGFHVCQPLSPIRNRTPFASAENHNYFPVNSAHEFAVREAEDTTLTSQPQGAPLYTDLLLNPQGNLSLVDSSSSSPSRMNSEGHLHVSGSLQENTPFTFFAVSHFPNQNDNGSRMAASGFTDEKETSKIKADPEKLKKLQESLLEEDSEEEGDLCRICQIAGGSPSNPLLEPCGCVGSLQFVHQECLKKWLKVKITSGADLGAVKTCEMCKQGLLVDLGDFNMIEFYQKHQQSQAQNELMNSGLYLVLLLHLYEQRFAELMRLNHNQVERERLSRNYPQPRTEENENSELGDGNEGSISQSQVV",
# MARHB_HUMAN
"MSFEGGHGGSRCRGAESGDAEPPPQPPPPPPPTPPPGEPAPVPAAPRYLPPLPASPETPERAAGPSEPLGEVAPRCRGADELPPPPLPLQPAGQEVAAAGDSGEGPRRLPEAAAAKGGPGESEAGAGGERERRGAGDQPETRSVCSSRSSSSGGGDQRAGHQHQHHQPICKICFQGAEQGELLNPCRCDGSVRYTHQLCLLKWISERGSWTCELCCYRYHVIAIKMKQPCQWQSISITLVEKVQMIAVILGSLFLIASVTWLLWSAFSPYAVWQRKDILFQICYGMYGFMDLVCIGLIVHEGAAVYRVFKRWRAVNLHWDVLNYDKATDIEESSRGESSTSRTLWLPLTALRNRNLVHPTQLTSPRFQCGYVLLHLFNRMRPHEDLSEDNSSGEVVMRVTSV",
# MARH2_HUMAN
"MTTGDCCHLPGSLCDCSGSPAFSKVVEATGLGPPQYVAQVTSRDGRLLSTVIRALDTPSDGPFCRICHEGANGECLLSPCGCTGTLGAVHKSCLEKWLSSSNTSYCELCHTEFAVEKRPRPLTEWLKDPGPRTEKRTLCCDMVCFLFITPLAAISGWLCLRGAQDHLRLHSQLEAVGLIALTIALFTIYVLWTLVSFRYHCQLYSEWRKTNQKVRLKIREADSPEGPQHSPLAAGLLKKVAEETPV",
# MARH3_HUMAN
"MTTSRCSHLPEVLPDCTSSAAPVVKTVEDCGSLVNGQPQYVMQVSAKDGQLLSTVVRTLATQSPFNDRPMCRICHEGSSQEDLLSPCECTGTLGTIHRSCLEHWLSSSNTSYCELCHFRFAVERKPRPLVEWLRNPGPQHEKRTLFGDMVCFLFITPLATISGWLCLRGAVDHLHFSSRLEAVGLIALTVALFTIYLFWTLVSFRYHCRLYNEWRRTNQRVILLIPKSVNVPSNQPSLLGLHSVKRNSKETVV",
# MARH4_HUMAN
"MLMPLCGLLWWWWCCCSGWYCYGLCAPAPQMLRHQGLLKCRCRMLFNDLKVFLLRRPPQAPLPMHGDPQPPGLAANNTLPALGAGGWAGWRGPREVVGREPPPVPPPPPLPPSSVEDDWGGPATEPPASLLSSASSDDFCKEKTEDRYSLGSSLDSGMRTPLCRICFQGPEQGELLSPCRCDGSVKCTHQPCLIKWISERGCWSCELCYYKYHVIAISTKNPLQWQAISLTVIEKVQVAAAILGSLFLIASISWLIWSTFSPSARWQRQDLLFQICYGMYGFMDVVCIGLIIHEGPSVYRIFKRWQAVNQQWKVLNYDKTKDLEDQKAGGRTNPRTSSSTQANIPSSEEETAGTPAPEQGPAQAAGHPSGPLSHHHCAYTILHILSHLRPHEQRSPPGSSRELVMRVTTV",
# MARH5_HUMAN
"MPDQALQQMLDRSCWVCFATDEDDRTAEWVRPCRCRGSTKWVHQACLQRWVDEKQRGNSTARVACPQCNAEYLIVFPKLGPVVYVLDLADRLISKACPFAAAGIMVGSIYWTAVTYGAVTVMQVVGHKEGLDVMERADPLFLLIGLPTIPVMLILGKMIRWEDYVLRLWRKYSNKLQILNSIFPGIGCPVPRIPAEANPLADHVSATRILCGALVFPTIATIVGKLMFSSVNSNLQRTILGGIAFVAIKGAFKVYFKQQQYLRQAHRKILNYPEQEEA",
# MARH6_HUMAN
"MDTAEEDICRVCRSEGTPEKPLYHPCVCTGSIKFIHQECLVQWLKHSRKEYCELCKHRFAFTPIYSPDMPSRLPIQDIFAGLVTSIGTAIRYWFHYTLVAFAWLGVVPLTACRIYKCLFTGSVSSLLTLPLDMLSTENLLADCLQGCFVVTCTLCAFISLVWLREQIVHGGAPIWLEHAAPPFNAAGHHQNEAPAGGNGAENVAADQPANPPAENAVVGENPDAQDDQAEEEEEDNEEEDDAGVEDAADANNGAQDDMNWNALEWDRAAEELTWERMLGLDGSLVFLEHVFWVVSLNTLFILVFAFCPYHIGHFSLVGLGFEEHVQASHFEGLITTIVGYILLAITLIICHGLATLVKFHRSRRLLGVCYIVVKVSLLVVVEIGVFPLICGWWLDICSLEMFDATLKDRELSFQSAPGTTMFLHWLVGMVYVFYFASFILLLREVLRPGVLWFLRNLNDPDFNPVQEMIHLPIYRHLRRFILSVIVFGSIVLLMLWLPIRIIKSVLPNFLPYNVMLYSDAPVSELSLELLLLQVVLPALLEQGHTRQWLKGLVRAWTVTAGYLLDLHSYLLGDQEENENSANQQVNNNQHARNNNAIPVVGEGLHAAHQAILQQGGPVGFQPYRRPLNFPLRIFLLIVFMCITLLIASLICLTLPVFAGRWLMSFWTGTAKIHELYTAACGLYVCWLTIRAVTVMVAWMPQGRRVIFQKVKEWSLMIMKTLIVAVLLAGVVPLLLGLLFELVIVAPLRVPLDQTPLFYPWQDWALGVLHAKIIAAITLMGPQWWLKTVIEQVYANGIRNIDLHYIVRKLAAPVISVLLLSLCVPYVIASGVVPLLGVTAEMQNLVHRRIYPFLLMVVVLMAILSFQVRQFKRLYEHIKNDKYLVGQRLVNYERKSGKQGSSPPPPQSSQE",
# MARH7_HUMAN
"MESKPSRIPRRISVQPSSSLSARMMSGSRGSSLNDTYHSRDSSFRLDSEYQSTSASASASPFQSAWYSESEITQGARSRSQNQQRDHDSKRPKLSCTNCTTSAGRNVGNGLNTLSDSSWRHSQVPRSSSMVLGSFGTDLMRERRDLERRTDSSISNLMDYSHRSGDFTTSSYVQDRVPSYSQGARPKENSMSTLQLNTSSTNHQLPSEHQTILSSRDSRNSLRSNFSSRESESSRSNTQPGFSYSSSRDEAPIISNSERVVSSQRPFQESSDNEGRRTTRRLLSRIASSMSSTFFSRRSSQDSLNTRSLNSENSYVSPRILTASQSRSNVPSASEVPDNRASEASQGFRFLRRRWGLSSLSHNHSSESDSENFNQESEGRNTGPWLSSSLRNRCTPLFSRRRREGRDESSRIPTSDTSSRSHIFRRESNEVVHLEAQNDPLGAAANRPQASAASSSATTGGSTSDSAQGGRNTGISGILPGSLFRFAVPPALGSNLTDNVMITVDIIPSGWNSADGKSDKTKSAPSRDPERLQKIKESLLLEDSEEEEGDLCRICQMAAASSSNLLIEPCKCTGSLQYVHQDCMKKWLQAKINSGSSLEAVTTCELCKEKLELNLEDFDIHELHRAHANEQAEYEFISSGLYLVVLLHLCEQSFSDMMGNTNEPSTRVRFINLARTLQAHMEDLETSEDDSEEDGDHNRTFDIA",
# MARH8_HUMAN
"MSMPLHQISAIPSQDAISARVYRSKTKEKEREEQNEKTLGHFMSHSSNISKAGSPPSASAPAPVSSFSRTSITPSSQDICRICHCEGDDESPLITPCHCTGSLHFVHQACLQQWIKSSDTRCCELCKYEFIMETKLKPLRKWEKLQMTSSERRKIMCSVTFHVIAITCVVWSLYVLIDRTAEEIKQGQATGILEWPFWTKLVVVAIGFTGGLLFMYVQCKVYVQLWKRLKAYNRVIYVQNCPETSKKNIFEKSPLTEPNFENKHGYGICHSDTNSSCCTEPEDTGAEIIHV",
# MARH9_HUMAN
"MLKSRLRMFLNELKLLVLTGGGRPRAEPQPRGGRGGGCGWAPFAGCSTRDGDGDEEEYYGSEPRARGLAGDKEPRAGPLPPPAPPLPPPGALDALSLSSSLDSGLRTPQCRICFQGPEQGELLSPCRCDGSVRCTHQPCLIRWISERGSWSCELCYFKYQVLAISTKNPLQWQAISLTVIEKVQIAAIVLGSLFLVASISWLIWSSLSPSAKWQRQDLLFQICYGMYGFMDVVCIGLIIHEGSSVYRIFKRWQAVNQQWKVLNYDKTKDIGGDAGGGTAGKSGPRNSRTGPTSGATSRPPAAQRMRTLLPQRCGYTILHLLGQLRPPDARSSSHSGREVVMRVTTV",
# MDM2_HUMAN
"MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKDTYTMKEVLFYLGQYIMTKRLYDEKQQHIVYCSNDLLGDLFGVPSFSVKEHRKIYTMIYRNLVVVNQQESSDSGTSVSENRCHLEGGSDQKDLVQELQEEKPSSSHLVSRPSTSSRRRAISETEENSDELSGERQRKRHKSDSISLSFDESLALCVIREICCERSSSSESTGTPSNPDLDAGVSEHSGDWLDQDSVSDQFSVEFEVESLDSEDYSLSEEGQELSDEDDEVYQVTVYQAGESDTDSFEEDPEISLADYWKCTSCNEMNPPLPSHCNRCWALRENWLPEDKGKDKGEISEKAKLENSTQAEEGFDVPDCKKTIVNDSRESCVEENDDKITQASQSQESEDYSQPSTSSSIIYSSQEDVKEFEREETQDKEESVESSLPLNAIEPCVICQGRPKNGCIVHGKTGHLMACFTCAKKLKKRNKPCPVCRQPIQMIVLTYFP",
# MDM4_HUMAN
"MTSFSTSAQCSTSDSACRISPGQINQVRPKLPLLKILHAAGAQGEMFTVKEVMHYLGQYIMVKQLYDQQEQHMVYCGGDLLGELLGRQSFSVKDPSPLYDMLRKNLVTLATATTDAAQTLALAQDHSMDIPSQDQLKQSAEESSTSRKRTTEDDIPTLPTSEHKCIHSREDEDLIENLAQDETSRLDLGFEEWDVAGLPWWFLGNLRSNYTPRSNGSTDLQTNQDVGTAIVSDTTDDLWFLNESVSEQLGVGIKVEAADTEQTSEEVGKVSDKKVIEVGKNDDLEDSKSLSDDTDVEVTSEDEWQCTECKKFNSPSKRYCFRCWALRKDWYSDCSKLTHSLSTSDITAIPEKENEGNDVPDCRRTISAPVVRPKDAYIKKENSKLFDPCNSVEFLDLAHSSESQETISSMGEQLDNLSEQRTDTENMEDCQNLLKPCSLCEKRPRDGNIIHGRTGHLVTCFHCARRLKKAGASCPICKKEIQLVIKVFIA",
# MECOM_HUMAN
"MRSKGRARKLATNNECVYGNYPEIPLEEMPDADGVASTPSLNIQEPCSPATSSEAFTPKEGSPYKAPIYIPDDIPIPAEFELRESNMPGAGLGIWTKRKIEVGEKFGPYVGEQRSNLKDPSYGWEILDEFYNVKFCIDASQPDVGSWLKYIRFAGCYDQHNLVACQINDQIFYRVVADIAPGEELLLFMKSEDYPHETMAPDIHEERQYRCEDCDQLFESKAELADHQKFPCSTPHSAFSMVEEDFQQKLESENDLQEIHTIQECKECDQVFPDLQSLEKHMLSHTEEREYKCDQCPKAFNWKSNLIRHQMSHDSGKHYECENCAKVFTDPSNLQRHIRSQHVGARAHACPECGKTFATSSGLKQHKHIHSSVKPFICEVCHKSYTQFSNLCRHKRMHADCRTQIKCKDCGQMFSTTSSLNKHRRFCEGKNHFAAGGFFGQGISLPGTPAMDKTSMVNMSHANPGLADYFGANRHPAGLTFPTAPGFSFSFPGLFPSGLYHRPPLIPASSPVKGLSSTEQTNKSQSPLMTHPQILPATQDILKALSKHPSVGDNKPVELQPERSSEERPFEKISDQSESSDLDDVSTPSGSDLETTSGSDLESDIESDKEKFKENGKMFKDKVSPLQNLASINNKKEYSNHSIFSPSLEEQTAVSGAVNDSIKAIASIAEKYFGSTGLVGLQDKKVGALPYPSMFPLPFFPAFSQSMYPFPDRDLRSLPLKMEPQSPGEVKKLQKGSSESPFDLTTKRKDEKPLTPVPSKPPVTPATSQDQPLDLSMGSRSRASGTKLTEPRKNHVFGGKKGSNVESRPASDGSLQHARPTPFFMDPIYRVEKRKLTDPLEALKEKYLRPSPGFLFHPQMSAIENMAEKLESFSALKPEASELLQSVPSMFNFRAPPNALPENLLRKGKERYTCRYCGKIFPRSANLTRHLRTHTGEQPYRCKYCDRSFSISSNLQRHVRNIHNKEKPFKCHLCDRCFGQQTNLDRHLKKHENGNMSGTATSSPHSELESTGAILDDKEDAYFTEIRNFIGNSNHGSQSPRNVEERMNGSHFKDEKALVTSQNSDLLDDEEVEDEVLLDEEDEDNDITGKTGKEPVTSNLHEGNPEDDYEETSALEMSCKTSPVRYKEEEYKSGLSALDHIRHFTDSLKMRKMEDNQYSEAELSSFSTSHVPEELKQPLHRKSKSQAYAMMLSLSDKESLHSTSHSSSNVWHSMARAAAESSAIQSISHV",
# MEX3A_HUMAN
"MPSLVVSGIMERNGGFGELGCFGGSAKDRGLLEDERALQLALDQLCLLGLGEPPAPTAGEDGGGGGGGAPAQPAAPPQPAPPPPPAAPPAAPTAAPAAQTPQPPTAPKGASDAKLCALYKEAELRLKGSSNTTECVPVPTSEHVAEIVGRQGCKIKALRAKTNTYIKTPVRGEEPVFMVTGRREDVATARREIISAAEHFSMIRASRNKSGAAFGVAPALPGQVTIRVRVPYRVVGLVVGPKGATIKRIQQQTNTYIITPSRDRDPVFEITGAPGNVERAREEIETHIAVRTGKILEYNNENDFLAGSPDAAIDSRYSDAWRVHQPGCKPLSTFRQNSLGCIGECGVDSGFEAPRLGEQGGDFGYGGYLFPGYGVGKQDVYYGVAETSPPLWAGQENATPTSVLFSSASSSSSSSAKARAGPPGAHRSPATSAGPELAGLPRRPPGEPLQGFSKLGGGGLRSPGGGRDCMVCFESEVTAALVPCGHNLFCMECAVRICERTDPECPVCHITATQAIRIFS",
# MEX3B_HUMAN
"MPSSLFADLERNGSGGGGGGSSGGGETLDDQRALQLALDQLSLLGLDSDEGASLYDSEPRKKSVNMTECVPVPSSEHVAEIVGRQGCKIKALRAKTNTYIKTPVRGEEPVFVVTGRKEDVAMARREIISAAEHFSMIRASRNKNTALNGAVPGPPNLPGQTTIQVRVPYRVVGLVVGPKGATIKRIQQQTHTYIVTPSRDKEPVFEVTGMPENVDRAREEIEAHIALRTGGIIELTDENDFHANGTDVGFDLHHGSGGSGPGSLWSKPTPSITPTPGRKPFSSYRNDSSSSLGSASTDSYFGGGTSSSAAATQRLADYSPPSPALSFAHNGNNNNNGNGYTYTAGGEASVPSPDGCPELQPTFDPAPAPPPGAPLIWAQFERSPGGGPAAPVSSSCSSSASSSASSSSVVFPGGGASAPSNANLGLLVHRRLHPGTSCPRLSPPLHMAPGAGEHHLARRVRSDPGGGGLAYAAYANGLGAQLPGLQPSDTSGSSSSSSSSSSSSSSSSGLRRKGSRDCSVCFESEVIAALVPCGHNLFCMECANRICEKSEPECPVCHTAVTQAIRIFS",
# MEX3C_HUMAN
"MPSGSSAALALAAAPAPLPQPPPPPPPPPPPLPPPSGGPELEGDGLLLRERLAALGLDDPSPAEPGAPALRAPAAAAQGQARRAAELSPEERAPPGRPGAPEAAELELEEDEEEGEEAELDGDLLEEEELEEAEEEDRSSLLLLSPPAATASQTQQIPGGSLGSVLLPAARFDAREAAAAAAAAGVLYGGDDAQGMMAAMLSHAYGPGGCGAAAAALNGEQAALLRRKSVNTTECVPVPSSEHVAEIVGRQGCKIKALRAKTNTYIKTPVRGEEPIFVVTGRKEDVAMAKREILSAAEHFSMIRASRNKNGPALGGLSCSPNLPGQTTVQVRVPYRVVGLVVGPKGATIKRIQQQTHTYIVTPSRDKEPVFEVTGMPENVDRAREEIEMHIAMRTGNYIELNEENDFHYNGTDVSFEGGTLGSAWLSSNPVPPSRARMISNYRNDSSSSLGSGSTDSYFGSNRLADFSPTSPFSTGNFWFGDTLPSVGSEDLAVDSPAFDSLPTSAQTIWTPFEPVNPLSGFGSDPSGNMKTQRRGSQPSTPRLSPTFPESIEHPLARRVRSDPPSTGNHVGLPIYIPAFSNGTNSYSSSNGGSTSSSPPESRRKHDCVICFENEVIAALVPCGHNLFCMECANKICEKRTPSCPVCQTAVTQAIQIHS",
# MEX3D_HUMAN
"MPSSLGQPDGGGGGGGGGGGVGAAGEDPGPGPAPPPEGAQEAAPAPRPPPEPDDAAAALRLALDQLSALGLGGAGDTDEEGAAGDGAAAAGGADGGAAPEPVPPDGPEAGAPPTLAPAVAPGSLPLLDPNASPPPPPPPRPSPPDVFAGFAPHPAALGPPTLLADQMSVIGSRKKSVNMTECVPVPSSEHVAEIVGRQGCKIKALRAKTNTYIKTPVRGEEPVFIVTGRKEDVEMAKREILSAAEHFSIIRATRSKAGGLPGAAQGPPNLPGQTTIQVRVPYRVVGLVVGPKGATIKRIQQRTHTYIVTPGRDKEPVFAVTGMPENVDRAREEIEAHITLRTGAFTDAGPDSDFHANGTDVCLDLLGAAASLWAKTPNQGRRPPTATAGLRGDTALGAPSAPEAFYAGSRGGPSVPDPGPASPYSGSGNGGFAFGAEGPGAPVGTAAPDDCDFGFDFDFLALDLTVPAAATIWAPFERAAPLPAFSGCSTVNGAPGPPAAGARRSSGAGTPRHSPTLPEPGGLRLELPLSRRGAPDPVGALSWRPPQGPVSFPGGAAFSTATSLPSSPAAAACAPLDSGASENSRKPPSASSAPALARECVVCAEGEVMAALVPCGHNLFCMDCAVRICGKSEPECPACRTPATQAIHIFS",
# MGRN1_HUMAN
"MGSILSRRIAGVEDIDIQANSAYRYPPKSGNYFASHFFMGGEKFDTPHPEGYLFGENMDLNFLGSRPVQFPYVTPAPHEPVKTLRSLVNIRKDSLRLVRYKDDADSPTEDGDKPRVLYSLEFTFDADARVAITIYCQASEEFLNGRAVYSPKSPSLQSETVHYKRGVSQQFSLPSFKIDFSEWKDDELNFDLDRGVFPVVIQAVVDEGDVVEVTGHAHVLLAAFEKHMDGSFSVKPLKQKQIVDRVSYLLQEIYGIENKNNQETKPSDDENSDNSNECVVCLSDLRDTLILPCRHLCLCTSCADTLRYQANNCPICRLPFRALLQIRAVRKKPGALSPVSFSPVLAQSLEHDEHSCPFKKSKPHPASLASKKPKRETNSDSVPPGYEPISLLEALNGLRAVSPAIPSAPLYEEITYSGISDGLSQASCPLAAIDHILDSSRQKGRPQSKAPDSTLRSPSSPIHEEDEEKLSEDVDAPPPLGGAELALRESSSPESFITEEVDESSSPQQGTRAASIENVLQDSSPEHCGRGPPADIYLPALGPDSCSVGIDE",
# MIB1_HUMAN
"MSNSRNNRVMVEGVGARVVRGPDWKWGKQDGGEGHVGTVRSFESPEEVVVVWDNGTAANYRCSGAYDLRILDSAPTGIKHDGTMCDTCRQQPIIGIRWKCAECTNYDLCTVCYHGDKHHLRHRFYRITTPGSERVLLESRRKSKKITARGIFAGARVVRGVDWQWEDQDGGNGRRGKVTEIQDWSASSPHSAAYVLWDNGAKNLYRVGFEGMSDLKCVQDAKGGSFYRDHCPVLGEQNGNRNPGGLQIGDLVNIDLDLEIVQSLQHGHGGWTDGMFETLTTTGTVCGIDEDHDIVVQYPSGNRWTFNPAVLTKANIVRSGDAAQGAEGGTSQFQVGDLVQVCYDLERIKLLQRGHGEWAEAMLPTLGKVGRVQQIYSDSDLKVEVCGTSWTYNPAAVSKVASAGSAISNASGERLSQLLKKLFETQESGDLNEELVKAAANGDVAKVEDLLKRPDVDVNGQCAGHTAMQAASQNGHVDILKLLLKQNVDVEAEDKDGDRAVHHAAFGDEGAVIEVLHRGSADLNARNKRRQTPLHIAVNKGHLQVVKTLLDFGCHPSLQDSEGDTPLHDAISKKRDDILAVLLEAGADVTITNNNGFNALHHAALRGNPSAMRVLLSKLPRPWIVDEKKDDGYTALHLAALNNHVEVAELLVHQGNANLDIQNVNQQTALHLAVERQHTQIVRLLVRAGAKLDIQDKDGDTPLHEALRHHTLSQLRQLQDMQDVGKVDAAWEPSKNTLIMGLGTQGAEKKSAASIACFLAANGADLSIRNKKGQSPLDLCPDPNLCKALAKCHKEKVSGQVGSRSPSMISNDSETLEECMVCSDMKRDTLFGPCGHIATCSLCSPRVKKCLICKEQVQSRTKIEECVVCSDKKAAVLFQPCGHMCACENCANLMKKCVQCRAVVERRVPFIMCCGGKSSEDATDDISSGNIPVLQKDKDNTNVNADVQKLQQQLQDIKEQTMCPVCLDRLKNMIFLCGHGTCQLCGDRMSECPICRKAIERRILLY",
# MIB2_HUMAN
"MDPDPQAGVQVGMRVVRGVDWKWGQQDGGEGGVGTVVELGRHGSPSTPDRTVVVQWDQGTRTNYRAGYQGAHDLLLYDNAQIGVRHPNIICDCCKKHGLRGMRWKCRVCLDYDLCTQCYMHNKHELAHAFDRYETAHSRPVTLSPRQGLPRIPLRGIFQGAKVVRGPDWEWGSQDGGEGKPGRVVDIRGWDVETGRSVASVTWADGTTNVYRVGHKGKVDLKCVGEAAGGFYYKDHLPRLGKPAELQRRVSADSQPFQHGDKVKCLLDTDVLREMQEGHGGWNPRMAEFIGQTGTVHRITDRGDVRVQFNHETRWTFHPGALTKHHSFWVGDVVRVIGDLDTVKRLQAGHGEWTDDMAPALGRVGKVVKVFGDGNLRVAVAGQRWTFSPSCLVAYRPEEDANLDVAERARENKSSLSVALDKLRAQKSDPEHPGRLVVEVALGNAARALDLLRRRPEQVDTKNQGRTALQVAAYLGQVELIRLLLQARAGVDLPDDEGNTALHYAALGNQPEATRVLLSAGCRADAINSTQSTALHVAVQRGFLEVVRALCERGCDVNLPDAHSDTPLHSAISAGTGASGIVEVLTEVPNIDVTATNSQGFTLLHHASLKGHALAVRKILARARQLVDAKKEDGFTALHLAALNNHREVAQILIREGRCDVNVRNRKLQSPLHLAVQQAHVGLVPLLVDAGCSVNAEDEEGDTALHVALQRHQLLPLVADGAGGDPGPLQLLSRLQASGLPGSAELTVGAAVACFLALEGADVSYTNHRGRSPLDLAAEGRVLKALQGCAQRFRERQAGGGAAPGPRQTLGTPNTVTNLHVGAAPGPEAAECLVCSELALLVLFSPCQHRTVCEECARRMKKCIRCQVVVSKKLRPDGSEVASAAPAPGPPRQLVEELQSRYRQMEERITCPICIDSHIRLVFQCGHGACAPCGSALSACPICRQPIRDRIQIFV",
# TRI18_HUMAN
"METLESELTCPICLELFEDPLLLPCAHSLCFNCAHRILVSHCATNESVESITAFQCPTCRHVITLSQRGLDGLKRNVTLQNIIDRFQKASVSGPNSPSETRRERAFDANTMTSAEKVLCQFCDQDPAQDAVKTCVTCEVSYCDECLKATHPNKKPFTGHRLIEPIPDSHIRGLMCLEHEDEKVNMYCVTDDQLICALCKLVGRHRDHQVAALSERYDKLKQNLESNLTNLIKRNTELETLLAKLIQTCQHVEVNASRQEAKLTEECDLLIEIIQQRRQIIGTKIKEGKVMRLRKLAQQIANCKQCIERSASLISQAEHSLKENDHARFLQTAKNITERVSMATASSQVLIPEINLNDTFDTFALDFSREKKLLECLDYLTAPNPPTIREELCTASYDTITVHWTSDDEFSVVSYELQYTIFTGQANVVSLCNSADSWMIVPNIKQNHYTVHGLQSGTKYIFMVKAINQAGSRSSEPGKLKTNSQPFKLDPKSAHRKLKVSHDNLTVERDESSSKKSHTPERFTSQGSYGVAGNVFIDSGRHYWEVVISGSTWYAIGLAYKSAPKHEWIGKNSASWALCRCNNNWVVRHNSKEIPIEPAPHLRRVGILLDYDNGSIAFYDALNSIHLYTFDVAFAQPVCPTFTVWNKCLTIITGLPIPDHLDCTEQLP",
# TRIM1_HUMAN
"MGESPASVVLNASGGLFSLKMETLESELTCPICLELFEDPLLLPCAHSLCFSCAHRILVSSCSSGESIEPITAFQCPTCRYVISLNHRGLDGLKRNVTLQNIIDRFQKASVSGPNSPSESRRERTYRPTTAMSSERIACQFCEQDPPRDAVKTCITCEVSYCDRCLRATHPNKKPFTSHRLVEPVPDTHLRGITCLDHENEKVNMYCVSDDQLICALCKLVGRHRDHQVASLNDRFEKLKQTLEMNLTNLVKRNSELENQMAKLIQICQQVEVNTAMHEAKLMEECDELVEIIQQRKQMIAVKIKETKVMKLRKLAQQVANCRQCLERSTVLINQAEHILKENDQARFLQSAKNIAERVAMATASSQVLIPDINFNDAFENFALDFSREKKLLEGLDYLTAPNPPSIREELCTASHDTITVHWISDDEFSISSYELQYTIFTGQANFISKSWCSWGLWPEIRKCKEAVSCSRLAGAPRGLYNSVDSWMIVPNIKQNHYTVHGLQSGTRYIFIVKAINQAGSRNSEPTRLKTNSQPFKLDPKMTHKKLKISNDGLQMEKDESSLKKSHTPERFSGTGCYGAAGNIFIDSGCHYWEVVMGSSTWYAIGIAYKSAPKNEWIGKNASSWVFSRCNSNFVVRHNNKEMLVDVPPHLKRLGVLLDYDNNMLSFYDPANSLHLHTFDVTFILPVCPTFTIWNKSLMILSGLPAPDFIDYPERQECNCRPQESPYVSGMKTCH",
# MKRN1_HUMAN
"MAEAATPGTTATTSGAGAAAATAAAASPTPIPTVTAPSLGAGGGGGGSDGSGGGWTKQVTCRYFMHGVCKEGDNCRYSHDLSDSPYSVVCKYFQRGYCIYGDRCRYEHSKPLKQEEATATELTTKSSLAASSSLSSIVGPLVEMNTGEAESRNSNFATVGAGSEDWVNAIEFVPGQPYCGRTAPSCTEAPLQGSVTKEESEKEQTAVETKKQLCPYAAVGECRYGENCVYLHGDSCDMCGLQVLHPMDAAQRSQHIKSCIEAHEKDMELSFAVQRSKDMVCGICMEVVYEKANPSERRFGILSNCNHTYCLKCIRKWRSAKQFESKIIKSCPECRITSNFVIPSEYWVEEKEEKQKLILKYKEAMSNKACRYFDEGRGSCPFGGNCFYKHAYPDGRREEPQRQKVGTSSRYRAQRRNHFWELIEERENSNPFDNDEEEVVTFELGEMLLMLLAAGGDDELTDSEDEWDLFHDELEDFYDLDL",
# MKRN2_HUMAN
"MSTKQITCRYFMHGVCREGSQCLFSHDLANSKPSTICKYYQKGYCAYGTRCRYDHTRPSAAAGGAVGTMAHSVPSPAFHSPHPPSEVTASIVKTNSHEPGKREKRTLVLRDRNLSGMAERKTQPSMVSNPGSCSDPQPSPEMKPHSYLDAIRSGLDDVEASSSYSNEQQLCPYAAAGECRFGDACVYLHGEVCEICRLQVLHPFDPEQRKAHEKICMLTFEHEMEKAFAFQASQDKVCSICMEVILEKASASERRFGILSNCNHTYCLSCIRQWRCAKQFENPIIKSCPECRVISEFVIPSVYWVEDQNKKNELIEAFKQGMGKKACKYFEQGKGTCPFGSKCLYRHAYPDGRLAEPEKPRKQLSSQGTVRFFNSVRLWDFIENRESRHVPNNEDVDMTELGDLFMHLSGVESSEP",
# MKRN3_HUMAN
"MEEPAAPSEAHEAAGAQAGAEAAREGVSGPDLPVCEPSGESAAPDSALPHAARGWAPFPVAPVPAHLRRGGLRPAPASGGGAWPSPLPSRSSGIWTKQIICRYYIHGQCKEGENCRYSHDLSGRKMATEGGVSPPGASAGGGPSTAAHIEPPTQEVAEAPPAASSLSLPVIGSAAERGFFEAERDNADRGAAGGAGVESWADAIEFVPGQPYRGRWVASAPEAPLQSSETERKQMAVGSGLRFCYYASRGVCFRGESCMYLHGDICDMCGLQTLHPMDAAQREEHMRACIEAHEKDMELSFAVQRGMDKVCGICMEVVYEKANPNDRRFGILSNCNHSFCIRCIRRWRSARQFENRIVKSCPQCRVTSELVIPSEFWVEEEEEKQKLIQQYKEAMSNKACRYFAEGRGNCPFGDTCFYKHEYPEGWGDEPPGPGGGSFSAYWHQLVEPVRMGEGNMLYKSIKKELVVLRLASLLFKRFLSLRDELPFSEDQWDLLHYELEEYFNLIL",
# MKRN4_HUMAN
"MAEAAAPGTTVTTSGAGAAAAEAAETAEAVSPTPIPTVTAPSPRAGGGVGGSDGSDGSGGRGDSGAYDGSGACGGSDACDGSGDSSGDSWTKQVTCRYFKYGICKEGDNCRYSHDLSDRLCGVVCKYFQRGCCVYGDRCRCEHSKPLKQEEATATELTTKSSLAASSSLSSIVGPLVEMNTNEAESRNSNFATVVAGSEDWANAIEFVPGQPYCGRTVPSCTEAPLQGSVTKEESEEEQTAVETKKQLCPYAAVGQCRYGENCVYLHGDLCDMCGLQVLHPMDAAQRSQHIQACIEAHEKDMEFSFAVQRSKDKVCGICMEVVYEKANPNEHRFGILSNCNHTFCLKCIRKWRSAKEFESRIVKSCPQCRITSNFVIPSEYWVEEKEEKQKLIQKYKEAMSNKACKYFDEGRGSCPFGENCFYKHMYPDGRREEPQRQQVGTSSRNPGQQRNHFWEFFEEGANSNPFDDEEEAVTFELGEMLLML",
# MAT1_HUMAN
"MDDQGCPRCKTTKYRNPSLKLMVNVCGHTLCESCVDLLFVRGAGNCPECGTPLRKSNFRVQLFEDPTVDKEVEIRKKVLKIYNKREEDFPSLREYNDFLEEVEEIVFNLTNNVDLDNTKKKMEIYQKENKDVIQKNKLKLTREQEELEEALEVERQENEQRRLFIQKEEQLQQILKRKNKQAFLDELESSDLPVALLLAQHKDRSTQLEMQLEKPKPVKPVTFSTGIKMGQHISLAPIHKLEEALYEYQPLQIETYGPHVPELEMLGRLGYLNHVRAASPQDLAGGYTSSLACHRALQDAFSGLFWQPS",
# COAC_HUMAN
"MEPKASCPAAAPLMERKFHVLVGVTGSVAALKLPLLVSKLLDIPGLEVAVVTTERAKHFYSPQDIPVTLYSDADEWEIWKSRSDPVLHIDLRRWADLLLVAPLDANTLGKVASGICDNLLTCVMRAWDRSKPLLFCPAMNTAMWEHPITAQQVDQLKAFGYVEIPCVAKKLVCGDEGLGAMAEVGTIVDKVKEVLFQHSGFQQS",
# MUL1_HUMAN
"MESGGRPSLCQFILLGTTSVVTAALYSVYRQKARVSQELKGAKKVHLGEDLKSILSEAPGKCVPYAVIEGAVRSVKETLNSQFVENCKGVIQRLTLQEHKMVWNRTTHLWNDCSKIIHQRTNTVPFDLVPHEDGVDVAVRVLKPLDSVDLGLETVYEKFHPSIQSFTDVIGHYISGERPKGIQETEEMLKVGATLTGVGELVLDNNSVRLQPPKQGMQYYLSSQDFDSLLQRQESSVRLWKVLALVFGFATCATLFFILRKQYLQRQERLRLKQMQEEFQEHEAQLLSRAKPEDRESLKSACVVCLSSFKSCVFLECGHVCSCTECYRALPEPKKCPICRQAITRVIPLYNS",
# MYCB2_HUMAN
"MMMCAATASPAAASSGLGGDGFYPAATFSSSPAPGALFMPVPDGSVAAAGLGLGLPAADSRGHYQLLLSGRALADRYRRIYTAALNDRDQGGGSAGHPASRNKKILNKKKLKRKQKSKSKVKTRSKSENLENTVIIPDIKLHSNPSAFNIYCNVRHCVLEWQKKEISLAAASKNSVQSGESDSDEEEESKEPPIKLPKIIEVGLCEVFELIKETRFSHPSLCLRSLQALLNVLQGQQPEGLQSEPPEVLESLFQLLLEITVRSTGMNDSTGQSLTALSCACLFSLVASWGETGRTLQAISAILTNNGSHACQTIQVPTILNSLQRSVQAVLVGKIQIQDWFSNGIKKAALMHKWPLKEISVDEDDQCLLQNDGFFLYLLCKDGLYKIGSGYSGTVRGHIYNSTSRIRNRKEKKSWLGYAQGYLLYRDVNNHSMTAIRISPETLEQDGTVMLPDCHTEGQNILFTDGEYINQIAASRDDGFVVRIFATSTEPVLQQELQLKLARKCLHACGISLFDLEKDLHIISTGFDEESAILGAGREFALMKTANGKIYYTGKYQSLGIKQGGPSAGKWVELPITKSPKIVHFSVGHDGSHALLVAEDGSIFFTGSASKGEDGESTKSRRQSKPYKPKKIIKMEGKIVVYTACNNGSSSVISKDGELYMFGKDAIYSDSSSLVTDLKGHFVTQVAMGKAHTCVLMKNGEVWTFGVNNKGQCGRDTGAMNQGGKGFGVENMATAMDEDLEEELDEKDEKSMMCPPGMHKWKLEQCMVCTVCGDCTGYGASCVSSGRPDRVPGGICGCGSGESGCAVCGCCKACARELDGQEARQRGILDAVKEMIPLDLLLAVPVPGVNIEEHLQLRQEEKRQRVIRRHRLEEGRGPLVFAGPIFMNHREQALARLRSHPAQLKHKRDKHKDGSGERGEKDASKITTYPPGSVRFDCELRAVQVSCGFHHSVVLMENGDVYTFGYGQHGQLGHGDVNSRGCPTLVQALPGPSTQVTAGSNHTAVLLMDGQVFTFGSFSKGQLGRPILDVPYWNAKPAPMPNIGSKYGRKATWIGASGDQTFLRIDEALINSHVLATSEIFASKHIIGLVPASISEPPPFKCLLINKVDGSCKTFNDSEQEDLQGFGVCLDPVYDVIWRFRPNTRELWCYNAVVADARLPSAADMQSRCSILSPELALPTGSRALTTRSHAALHILGCLDTLAAMQDLKMGVASTEEETQAVMKVYSKEDYSVVNRFESHGGGWGYSAHSVEAIRFSADTDILLGGLGLFGGRGEYTAKIKLFELGPDGGDHETDGDLLAETDVLAYDCAAREKYAMMFDEPVLLQAGWWYVAWARVSGPSSDCGSHGQASITTDDGVVFQFKSSKKSNNGTDVNAGQIPQLLYRLPTSDGSASKGKQQTSEPVHILKRSFARTVSVECFESLLSILHWSWTTLVLGVEELRGLKGFQFTATLLDLERLRFVGTCCLRLLRVYTCEIYPVSATGKAVVEETSKLAECIGKTRTLLRKILSEGVDHCMVKLDNDPQGYLSQPLSLLEAVLQECHNTFTACFHSFYPTPALQWACLCDLLNCLDQDIQEANFKTSSSRLLAAVMSALCHTSVKLTSIFPIAYDGEVLLRSIVKQVSTENDSTLVHRFPLLVAHMEKLSQSEENISGMTSFREVLEKMLVIVVLPVRNSLRRENELFSSHLVSNTCGLLASIVSELTASALGSEVDGLNSLHSVKASANRFTKTSQGRSWNTGNGSPDAICFSVDKPGIVVVGFSVYGGGGIHEYELEVLVDDSEHAGDSTHSHRWTSLELVKGTYTTDDSPSDIAEIRLDKVVPLKENVKYAVRLRNYGSRTANGDGGMTTVQCPDGVTFTFSTCSLSSNGTNQTRGQIPQILYYRSEFDGDLQSQLLSKANEEDKNCSRALSVVSTVVRASKDLLHRALAVDADDIPELLSSSSLFSMLLPLIIAYIGPVAAAIPKVAVEVFGLVQQLLPSVAILNQKYAPPAFNPNQSTDSTTGNQPEQGLSACTTSSHYAVIESEHPYKPACVMHYKVTFPECVRWMTIEFDPQCGTAQSEDVLRLLIPVRTVQNSGYGPKLTSVHENLNSWIELKKFSGSSGWPTMVLVLPGNEALFSLETASDYVKDDKASFYGFKCFAIGYEFSPGPDEGVIQLEKELANLGGVCAAALMKKDLALPIGNELEEDLEILEEAALQVCKTHSGILGKGLALSHSPTILEALEGNLPLQIQSNEQSFLDDFIACVPGSSGGRLARWLQPDSYADPQKTSLILNKDDIRCGWPTTITVQTKDQYGDVVHVPNMKVEVKAVPVSQKKMSLQQDQAKKPQRIPGSPAVTAASSNTDMTYGGLASPKLDVSYEPMIVKEARYIAITMMKVYENYSFEELRFASPTPKRPSENMLIRVNNDGTYCANWTPGAIGLYTLHVTIDGIEIDAGLEVKVKDPPKGMIPPGTQLVKPKSEPQPNKVRKFVAKDSAGLRIRSHPSLQSEQIGIVKVNGTITFIDEIHNDDGVWLRLNDETIKKYVPNMNGYTEAWCLSFNQHLGKSLLVPVDESKTNTDDFFKDINSCCPQEATMQEQDMPFLRGGPGMYKVVKTGPSGHNIRSCPNLRGIPIGMLVLGNKVKAVGEVTNSEGTWVQLDQNSMVEFCESDEGEAWSLARDRGGNQYLRHEDEQALLDQNSQTPPPSPFSVQAFNKGASCSAQGFDYGLGNSKGDRGNISTSSKPASTSGKSELSSKHSRSLKPDGRMSRTTADQKKPRGTESLSASESLILKSDAAKLRSDSHSRSLSPNHNTLQTLKSDGRMPSSSRAESPGPGSRLSSPKPKTLPANRSSPSGASSPRSSSPHDKNLPQKSTAPVKTKLDPPRERSKSDSYTLDPDTLRKKKMPLTEPLRGRSTSPKPKSVPKDSTDSPGSENRAPSPHVVQENLHSEVVEVCTSSTLKTNSLTDSTCDDSSEFKSVDEGSNKVHFSIGKAPLKDEQEMRASPKISRKCANRHTRPKKEKSSFLFKGDGSKPLEPAKQAMSPSVAECARAVFASFLWHEGIVHDAMACSSFLKFHPELSKEHAPIRSSLNSQQPTEEKETKLKNRHSLEISSALNMFNIAPHGPDISKMGSINKNKVLSMLKEPPLHEKCEDGKTETTFEMSMHNTMKSKSPLPLTLQHLVAFWEDISLATIKAASQNMIFPSPGSCAVLKKKECEKENKKSKKEKKKKEKAEVRPRGNLFGEMAQLAVGGPEKDTICELCGESHPYPVTYHMRQAHPGCGRYAGGQGYNSIGHFCGGWAGNCGDGGIGGSTWYLVCDRCREKYLREKQAAAREKVKQSRRKPMQVKTPRALPTMEAHQVIKANALFLLSLSSAAEPSILCYHPAKPFQSQLPSVKEGISEDLPVKMPCLYLQTLARHHHENFVGYQDDNLFQDEMRYLRSTSVPAPYISVTPDASPNVFEEPESNMKSMPPSLETSPITDTDLAKRTVFQRSYSVVASEYDKQHSILPARVKAIPRRRVNSGDTEVGSSLLRHPSPELSRLISAHSSLSKGERNFQWPVLAFVIQHHDLEGLEIAMKQALRKSACRVFAMEAFNWLLCNVIQTTSLHDILWHFVASLTPAPVEPEEEEDEENKTSKENSEQEKDTRVCEHPLSDIVIAGEAAHPLPHTFHRLLQTISDLMMSLPSGSSLQQMALRCWSLKFKQSDHQFLHQSNVFHHINNILSKSDDGDSEESFSISIQSGFEAMSQELCIVMCLKDLTSIVDIKTSSRPAMIGSLTDGSTETFWESGDEDKNKTKNITINCVKGINARYVSVHVDNSRDLGNKVTSMTFLTGKAVEDLCRIKQVDLDSRHIGWVTSELPGGDNHIIKIELKGPENTLRVRQVKVLGWKDGESTKIAGQISASVAQQRNCEAETLRVFRLITSQVFGKLISGDAEPTPEQEEKALLSSPEGEEKVYNATSDADLKEHMVGIIFSRSKLTNLQKQVCAHIVQAIRMEATRVREEWEHAISSKENANSQPNDEDASSDAYCFELLSMVLALSGSNVGRQYLAQQLTLLQDLFSLLHTASPRVQRQVTSLLRRVLPEVTPSRLASIIGVKSLPPADISDIIHSTEKGDWNKLGILDMFLGCIAKALTVQLKAKGTTITGTAGTTVGKGVTTVTLPMIFNSSYLRRGESHWWMKGSTPTQISEIIIKLIKDMAAGHLSEAWSRVTKNAIAETIIALTKMEEEFRSPVRCIATTRLWLALASLCVLDQDHVDRLSSGRWMGKDGQQKQMPMCDNHDDGETAAIILCNVCGNLCTDCDRFLHLHRRTKTHQRQVFKEEEEAIKVDLHEGCGRTKLFWLMALADSKTMKAMVEFREHTGKPTTSSSEACRFCGSRSGTELSAVGSVCSDADCQEYAKIACSKTHPCGHPCGGVKNEEHCLPCLHGCDKSATSLKQDADDMCMICFTEALSAAPAIQLDCSHIFHLQCCRRVLENRWLGPRITFGFISCPICKNKINHIVLKDLLDPIKELYEDVRRKALMRLEYEGLHKSEAITTPGVRFYNDPAGYAMNRYAYYVCYKCRKAYFGGEARCDAEAGRGDDYDPRELICGACSDVSRAQMCPKHGTDFLEYKCRYCCSVAVFFCFGTTHFCNACHDDFQRMTSIPKEELPHCPAGPKGKQLEGTECPLHVVHPPTGEEFALGCGVCRNAHTF",
# MYLIP_HUMAN
"MLCYVTRPDAVLMEVEVEAKANGEDCLNQVCRRLGIIEVDYFGLQFTGSKGESLWLNLRNRISQQMDGLAPYRLKLRVKFFVEPHLILQEQTRHIFFLHIKEALLAGHLLCSPEQAVELSALLAQTKFGDYNQNTAKYNYEELCAKELSSATLNSIVAKHKELEGTSQASAEYQVLQIVSAMENYGIEWHSVRDSEGQKLLIGVGPEGISICKDDFSPINRIAYPVVQMATQSGKNVYLTVTKESGNSIVLLFKMISTRAASGLYRAITETHAFYRCDTVTSAVMMQYSRDLKGHLASLFLNENINLGKKYVFDIKRTSKEVYDHARRALYNAGVVDLVSRNNQSPSHSPLKSSESSMNCSSCEGLSCQQTRVLQEKLRKLKEAMLCMVCCEEEINSTFCPCGHTVCCESCAAQLQSCPVCRSRVEHVQHVYLPTHTSLLNLTVI",
# NEDD4_HUMAN
"MAQSLRLHFAARRSNTYPLSETSGDDLDSHVHMCFKRPTRISTSNVVQMKLTPRQTALAPLIKENVQSQERSSVPSSENVNKKSSCLQISLQPTRYSGYLQSSNVLADSDDASFTCILKDGIYSSAVVDNELNAVNDGHLVSSPAICSGSLSNFSTSDNGSYSSNGSDFGSCASITSGGSYTNSVISDSSSYTFPPSDDTFLGGNLPSDSTSNRSVPNRNTTPCEIFSRSTSTDPFVQDDLEHGLEIMKLPVSRNTKIPLKRYSSLVIFPRSPSTTRPTSPTSLCTLLSKGSYQTSHQFIISPSEIAHNEDGTSAKGFLSTAVNGLRLSKTICTPGEVRDIRPLHRKGSLQKKIVLSNNTPRQTVCEKSSEGYSCVSVHFTQRKAATLDCETTNGDCKPEMSEIKLNSDSEYIKLMHRTSACLPSSQNVDCQININGELERPHSQMNKNHGILRRSISLGGAYPNISCLSSLKHNCSKGGPSQLLIKFASGNEGKVDNLSRDSNRDCTNELSNSCKTRDDFLGQVDVPLYPLPTENPRLERPYTFKDFVLHPRSHKSRVKGYLRLKMTYLPKTSGSEDDNAEQAEELEPGWVVLDQPDAACHLQQQQEPSPLPPGWEERQDILGRTYYVNHESRRTQWKRPTPQDNLTDAENGNIQLQAQRAFTTRRQISEETESVDNRESSENWEIIREDEATMYSNQAFPSPPPSSNLDVPTHLAEELNARLTIFGNSAVSQPASSSNHSSRRGSLQAYTFEEQPTLPVLLPTSSGLPPGWEEKQDERGRSYYVDHNSRTTTWTKPTVQATVETSQLTSSQSSAGPQSQASTSDSGQQVTQPSEIEQGFLPKGWEVRHAPNGRPFFIDHNTKTTTWEDPRLKIPAHLRGKTSLDTSNDLGPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRLENVAITGPAVPYSRDYKRKYEFFRRKLKKQNDIPNKFEMKLRRATVLEDSYRRIMGVKRADFLKARLWIEFDGEKGLDYGGVAREWFFLISKEMFNPYYGLFEYSATDNYTLQINPNSGLCNEDHLSYFKFIGRVAGMAVYHGKLLDGFFIRPFYKMMLHKPITLHDMESVDSEYYNSLRWILENDPTELDLRFIIDEELFGQTHQHELKNGGSEIVVTNKNKKEYIYLVIQWRFVNRIQKQMAAFKEGFFELIPQDLIKIFDENELELLMCGLGDVDVNDWREHTKYKNGYSANHQVIQWFWKAVLMMDSEKRIRLLQFVTGTSRVPMNGFAELYGSNGPQSFTVEQWGTPEKLPRAHTCFNRLDLPPYESFEELWDKLQMAIENTQGFDGVD",
# NED4L_HUMAN
"MATGLGEPVYGLSEDEGESRILRVKVVSGIDLAKKDIFGASDPYVKLSLYVADENRELALVQTKTIKKTLNPKWNEEFYFRVNPSNHRLLFEVFDENRLTRDDFLGQVDVPLSHLPTEDPTMERPYTFKDFLLRPRSHKSRVKGFLRLKMAYMPKNGGQDEENSDQRDDMEHGWEVVDSNDSASQHQEELPPPPLPPGWEEKVDNLGRTYYVNHNNRTTQWHRPSLMDVSSESDNNIRQINQEAAHRRFRSRRHISEDLEPEPSEGGDVPEPWETISEEVNIAGDSLGLALPPPPASPGSRTSPQELSEELSRRLQITPDSNGEQFSSLIQREPSSRLRSCSVTDAVAEQGHLPPPSAPAGRARSSTVTGGEEPTPSVAYVHTTPGLPSGWEERKDAKGRTYYVNHNNRTTTWTRPIMQLAEDGASGSATNSNNHLIEPQIRRPRSLSSPTVTLSAPLEGAKDSPVRRAVKDTLSNPQSPQPSPYNSPKPQHKVTQSFLPPGWEMRIAPNGRPFFIDHNTKTTTWEDPRLKFPVHMRSKTSLNPNDLGPLPPGWEERIHLDGRTFYIDHNSKITQWEDPRLQNPAITGPAVPYSREFKQKYDYFRKKLKKPADIPNRFEMKLHRNNIFEESYRRIMSVKRPDVLKARLWIEFESEKGLDYGGVAREWFFLLSKEMFNPYYGLFEYSATDNYTLQINPNSGLCNEDHLSYFTFIGRVAGLAVFHGKLLDGFFIRPFYKMMLGKQITLNDMESVDSEYYNSLKWILENDPTELDLMFCIDEENFGQTYQVDLKPNGSEIMVTNENKREYIDLVIQWRFVNRVQKQMNAFLEGFTELLPIDLIKIFDENELELLMCGLGDVDVNDWRQHSIYKNGYCPNHPVIQWFWKAVLLMDAEKRIRLLQFVTGTSRVPMNGFAELYGSNGPQLFTIEQWGSPEKLPRAHTCFNRLDLPPYETFEDLREKLLMAVENAQGFEGVD",
# NEUL1_HUMAN
"MGNNFSSIPSLPRGNPSRAPRGHPQNLKDSIGGPFPVTSHRCHHKQKHCPAVLPSGGLPATPLLFHPHTKGSQILMDLSHKAVKRQASFCNAITFSNRPVLIYEQVRLKITKKQCCWSGALRLGFTSKDPSRIHPDSLPKYACPDLVSQSGFWAKALPEEFANEGNIIAFWVDKKGRVFHRINDSAVMLFFSGVRTADPLWALVDVYGLTRGVQLLDSELVLPDCLRPRSFTALRRPSLRREADDARLSVSLCDLNVPGADGDEAAPAAGCPIPQNSLNSQHSRALPAQLDGDLRFHALRAGAHVRILDEQTVARVEHGRDERALVFTSRPVRVAETIFVKVTRSGGARPGALSFGVTTCDPGTLRPADLPFSPEALVDRKEFWAVCRVPGPLHSGDILGLVVNADGELHLSHNGAAAGMQLCVDASQPLWMLFGLHGTITQIRILGSTILAERGIPSLPCSPASTPTSPSALGSRLSDPLLSTCSSGPLGSSAGGTAPNSPVSLPESPVTPGLGQWSDECTICYEHAVDTVIYTCGHMCLCYACGLRLKKALHACCPICRRPIKDIIKTYRSS",
# NEU1B_HUMAN
"MGNTVHRTLPDPSPPARLLATRPCCGPGPERRPVLGEAPRFHAQAKGKNVRLDGHSRRATRRNSFCNGVTFTQRPIRLYEQVRLRLVAVRPGWSGALRFGFTAHDPSLMSAQDIPKYACPDLVTRPGYWAKALPENLALRDTVLAYWADRHGRVFYSVNDGEPVLFHCGVAVGGPLWALIDVYGITDEVQLLESAFADTLTPARLSQARFSACLPPSSHDAANFDNNELENNQVVAKLGHLALGRAPGPPPADAAAAAIPCGPRERPRPASSPALLEADLRFHATRGPDVSLSADRKVACAPRPDGGRTLVFSERPLRPGESLFVEVGRPGLAAPGALAFGITSCDPGVLRPNELPADPDALLDRKEYWVVARAGPVPSGGDALSFTLRPGGDVLLGINGRPRGRLLCVDTTQALWAFFAVRGGVAGQLRLLGTLQSSPATTTPSGSLSGSQDDSDSDMTFSVNQSSSASESSLVTAPSSPLSPPVSPVFSPPEPAGIKNGECTVCFDGEVDTVIYTCGHMCLCHSCGLRLKRQARACCPICRRPIKDVIKIYRP",
# NEUL3_HUMAN
"MGAQLCFEANAKAPREALRFHAEAKGAQVRLDTRGCIAHRRTTFHDGIVFSQRPVRLGERVALRVLREESGWCGGLRVGFTRLDPACVSVPSLPPFLCPDLEEQSPTWAAVLPEGCALTGDLVRFWVDRRGCLFAKVNAGCRLLLREGVPVGAPLWAVMDVYGTTKAIELLDPTASRLPTPMPWDLSNKAVPEPKATPGEECAICFYHAANTRLVPCGHTYFCRYCAWRVFSDTAKCPVCRWQIEAVAPAQGPPALRVEEGS",
# NFX1_HUMAN
"MAEAPPVSGTFKFNTDAAEFIPQEKKNSGLNCGTQRRLDSNRIGRRNYSSPPPCHLSRQVPYDEISAVHQHSYHPSGSKPKSQQTSFQSSPCNKSPKSHGLQNQPWQKLRNEKHHIRVKKAQSLAEQTSDTAGLESSTRSESGTDLREHSPSESEKEVVGADPRGAKPKKATQFVYSYGRGPKVKGKLKCEWSNRTTPKPEDAGPESTKPVGVFHPDSSEASSRKGVLDGYGARRNEQRRYPQKRPPWEVEGARPRPGRNPPKQEGHRHTNAGHRNNMGPIPKDDLNERPAKSTCDSENLAVINKSSRRVDQEKCTVRRQDPQVVSPFSRGKQNHVLKNVETHTGSLIEQLTTEKYECMVCCELVRVTAPVWSCQSCYHVFHLNCIKKWARSPASQADGQSGWRCPACQNVSAHVPNTYTCFCGKVKNPEWSRNEIPHSCGEVCRKKQPGQDCPHSCNLLCHPGPCPPCPAFMTKTCECGRTRHTVRCGQAVSVHCSNPCENILNCGQHQCAELCHGGQCQPCQIILNQVCYCGSTSRDVLCGTDVGKSDGFGDFSCLKICGKDLKCGNHTCSQVCHPQPCQQCPRLPQLVRCCPCGQTPLSQLLELGSSSRKTCMDPVPSCGKVCGKPLPCGSLDFIHTCEKLCHEGDCGPCSRTSVISCRCSFRTKELPCTSLKSEDATFMCDKRCNKKRLCGRHKCNEICCVDKEHKCPLICGRKLRCGLHRCEEPCHRGNCQTCWQASFDELTCHCGASVIYPPVPCGTRPPECTQTCARVHECDHPVYHSCHSEEKCPPCTFLTQKWCMGKHEFRSNIPCHLVDISCGLPCSATLPCGMHKCQRLCHKGECLVDEPCKQPCTTPRADCGHPCMAPCHTSSPCPVTACKAKVELQCECGRRKEMVICSEASSTYQRIAAISMASKITDMQLGGSVEISKLITKKEVHQARLECDEECSALERKKRLAEAFHISEDSDPFNIRSSGSKFSDSLKEDARKDLKFVSDVEKEMETLVEAVNKGKNSKKSHSFPPMNRDHRRIIHDLAQVYGLESVSYDSEPKRNVVVTAIRGKSVCPPTTLTGVLEREMQARPPPPIPHHRHQSDKNPGSSNLQKITKEPIIDYFDVQD",
# NFXL1_HUMAN
"MEASWRQVAGGRGRSRGRATAAPSGNGVHLRGAGGGREKGSVGAVPSGTSPGGVATTAAAGSRHSPAGSQALQTTAASELMSQKKFEEIKKANQAAARKLVEEQFSSSSEEGDEDFEGKQGKILANTFITYTTQTDGDTRELERTKQYVNEAFQAGAMTCLICIASVKRNQAVWSCSGCFCIFHMPCIQKWAKDSQFLVSSVTDDDFGKKDCPWPCPKCRFEYKRSETPSRYYCYCGKVEDPPLDPWLVPHSCGQVCEREFKPPCGHKCLLLCHPGPCPPCPKMVTTTCYCKKAKPIPRRCSAKEWSCQLPCGQKLLCGQHKCENPCHAGSCQPCPRVSRQKCVCGKKVAERSCASPLWHCDQVCGKTLPCGNHTCEQVCHVGACGECPRSGKRFCPCQKSKFSLPCTEDVPTCGDSCDKVLECGIHRCSQRCHRGPCETCRQEVEKHCRCGKHTKRMPCHKPYLCETKCVKMRDCQKHQCRRKCCPGNCPPCDQNCGRTLGCRNHKCPSVCHRGSCYPCPETVDVKCNCGNTKVTVPCGRERTTRPPKCKEQCSRPPTCHHTSQEKHRCHFGSCPPCHQPCQKVLEKCGHLCPAPCHDQALIKQTGRHQPTGPWEQPSEPAFIQTALPCPPCQVPIPMECLGKHEVSPLPCHAVGPYSCKRVCGRILDCQNHTCMKECHKVTKTDGCTGKNKAGPECLHCEEGCSKSRPLGCLHPCILRCHPGECPPCVQMLRIKCHCKITSLYVECRKITTADVNEKNLLSCCKNQCPKELPCGHRCKEMCHPGECPFNCNQKVKLRCPCKRIKKELQCNKVRENQVSIECDTTCKEMKRKASEIKEAEAKAALEEEKRRQQAELEAFENRLKGRRKKNRKRDEVAVELSLWQKHKYYLISVCGVVVVVFAWYITHDVN",
# NHLC1_HUMAN
"MAAEASESGPALHELMREAEISLLECKVCFEKFGHRQQRRPRNLSCGHVVCLACVAALAHPRTLALECPFCRRACRGCDTSDCLPVLHLIELLGSALRQSPAAHRAAPSAPGALTCHHTFGGWGTLVNPTGLALCPKTGRVVVVHDGRRRVKIFDSGGGCAHQFGEKGDAAQDIRYPVDVTITNDCHVVVTDAGDRSIKVFDFFGQIKLVIGGQFSLPWGVETTPQNGIVVTDAEAGSLHLLDVDFAEGVLRRTERLQAHLCNPRGVAVSWLTGAIAVLEHPLALGTGVCSTRVKVFSSSMQLVGQVDTFGLSLYFPSKITASAVTFDHQGNVIVADTSGPAILCLGKPEEFPVPKPMVTHGLSHPVALTFTKENSLLVLDTASHSIKVYKVDWG",
# NOSIP_HUMAN
"MTRHGKNCTAGAVYTYHEKKKDTAASGYGTQNIRLSRDAVKDFDCCCLSLQPCHDPVVTPDGYLYEREAILEYILHQKKEIARQMKAYEKQRGTRREEQKELQRAASQDHVRGFLEKESAIVSRPLNPFTAKALSGTSPDDVQPGPSVGPPSKDKDKVLPSFWIPSLTPEAKATKLEKPSRTVTCPMSGKPLRMSDLTPVHFTPLDSSVDRVGLITRSERYVCAVTRDSLSNATPCAVLRPSGAVVTLECVEKLIRKDMVDPVTGDKLTDRDIIVLQRGGTGFAGSGVKLQAEKSRPVMQA",
# NSE1_HUMAN
"MQGSTRRMGVMTDVHRRFLQLLMTHGVLEEWDVKRLQTHCYKVHDRNATVDKLEDFINNINSVLESLYIEIKRGVTEDDGRPIYALVNLATTSISKMATDFAENELDLFRKALELIIDSETGFASSTNILNLVDQLKGKKMRKKEAEQVLQKFVQNKWLIEKEGEFTLHGRAILEMEQYIRETYPDAVKICNICHSLLIQGQSCETCGIRMHLPCVAKYFQSNAEPRCPHCNDYWPHEIPKVFDPEKERESGVLKSNKKSLRSRQH",
# PRKN_HUMAN
"MIVFVRFNSSHGFPVEVDSDTSIFQLKEVVAKRQGVPADQLRVIFAGKELRNDWTVQNCDLDQQSIVHIVQRPWRKGQEMNATGGDDPRNAAGGCEREPQSLTRVDLSSSVLPGDSVGLAVILHTDSRKDSPPAGSPAGRSIYNSFYVYCKGPCQRVQPGKLRVQCSTCRQATLTLTQGPSCWDDVLIPNRMSGECQSPHCPGTSAEFFFKCGAHPTSDKETSVALHLIATNSRNITCITCTDVRSPVLVFQCNSRHVICLDCFHLYCVTRLNDRQFVHDPQLGYSLPCVAGCPNSLIKELHHFRILGEEQYNRYQQYGAEECVLQMGGVLCPRPGCGAGLLPEPDQRKVTCEGGNGLGCGFAFCRECKEAYHEGECSAVFEASGTTTQAYRVDERAAEQARWEAASKETIKKTTKPCPRCHVPVEKNGGCMHMKCPQPQCRLEWCWNCGCEWNRVCMGDHWFDV",
# PCGF1_HUMAN
"MASPQGGQIAIAMRLRNQLQSVYKMDPLRNEEEVRVKIKDLNEHIVCCLCAGYFVDATTITECLHTFCKSCIVKYLQTSKYCPMCNIKIHETQPLLNLKLDRVMQDIVYKLVPGLQDSEEKRIREFYQSRGLDRVTQPTGEEPALSNLGLPFSSFDHSKAHYYRYDEQLNLCLERLSSGKDKNKSVLQNKYVRCSVRAEVRHLRRVLCHRLMLNPQHVQLLFDNEVLPDHMTMKQIWLSRWFGKPSPLLLQYSVKEKRR",
# PCGF2_HUMAN
"MHRTTRIKITELNPHLMCALCGGYFIDATTIVECLHSFCKTCIVRYLETNKYCPMCDVQVHKTRPLLSIRSDKTLQDIVYKLVPGLFKDEMKRRRDFYAAYPLTEVPNGSNEDRGEVLEQEKGALSDDEIVSLSIEFYEGARDRDEKKGPLENGDGDKEKTGVRFLRCPAAMTVMHLAKFLRNKMDVPSKYKVEVLYEDEPLKEYYTLMDIAYIYPWRRNGPLPLKYRVQPACKRLTLATVPTPSEGTNTSGASECESVSDKAPSPATLPATSSSLPSPATPSHGSPSSHGPPATHPTSPTPPSTASGATTAANGGSLNCLQTPSSTSRGRKMTVNGAPVPPLT",
# PCGF3_HUMAN
"MLTRKIKLWDINAHITCRLCSGYLIDATTVTECLHTFCRSCLVKYLEENNTCPTCRIVIHQSHPLQYIGHDRTMQDIVYKLVPGLQEAEMRKQREFYHKLGMEVPGDIKGETCSAKQHLDSHRNGETKADDSSNKEAAEEKPEEDNDYHRSDEQVSICLECNSSKLRGLKRKWIRCSAQATVLHLKKFIAKKLNLSSFNELDILCNEEILGKDHTLKFVVVTRWRFKKAPLLLHYRPKMDLL",
# PCGF5_HUMAN
"MATQRKHLVKDFNPYITCYICKGYLIKPTTVTECLHTFCKTCIVQHFEDSNDCPRCGNQVHETNPLEMLRLDNTLEEIIFKLVPGLREQELERESEFWKKNKPQENGQDDTSKADKPKVDEEGDENEDDKDYHRSDPQIAICLDCLRNNGQSGDNVVKGLMKKFIRCSTRVTVGTIKKFLSLKLKLPSSYELDVLCNGEIMGKDHTMEFIYMTRWRLRGENFRCLNCSASQVCSQDGPLYQSYPMVLQYRPRIDFG",
# PCGF6_HUMAN
"MEGVAVVTAGSVGAAKTEGAAALPPPPPVSPPALTPAPAAGEEGPAPLSETGAPGCSGSRPPELEPERSLGRFRGRFEDEDEELEEEEELEEEEEEEEEDMSHFSLRLEGGRQDSEDEEERLINLSELTPYILCSICKGYLIDATTITECLHTFCKSCIVRHFYYSNRCPKCNIVVHQTQPLYNIRLDRQLQDIVYKLVINLEEREKKQMHDFYKERGLEVPKPAVPQPVPSSKGRSKKVLESVFRIPPELDMSLLLEFIGANEGTGHFKPLEKKFVRVSGEATIGHVEKFLRRKMGLDPACQVDIICGDHLLEQYQTLREIRRAIGDAAMQDGLLVLHYGLVVSPLKIT",
# PZRN3_HUMAN
"MGFELDRFDGDVDPDLKCALCHKVLEDPLTTPCGHVFCAGCVLPWVVQEGSCPARCRGRLSAKELNHVLPLKRLILKLDIKCAYATRGCGRVVKLQQLPEHLERCDFAPARCRHAGCGQVLLRRDVEAHMRDACDARPVGRCQEGCGLPLTHGEQRAGGHCCARALRAHNGALQARLGALHKALKKEALRAGKREKSLVAQLAAAQLELQMTALRYQKKFTEYSARLDSLSRCVAAPPGGKGEETKSLTLVLHRDSGSLGFNIIGGRPSVDNHDGSSSEGIFVSKIVDSGPAAKEGGLQIHDRIIEVNGRDLSRATHDQAVEAFKTAKEPIVVQVLRRTPRTKMFTPPSESQLVDTGTQTDITFEHIMALTKMSSPSPPVLDPYLLPEEHPSAHEYYDPNDYIGDIHQEMDREELELEEVDLYRMNSQDKLGLTVCYRTDDEDDIGIYISEIDPNSIAAKDGRIREGDRIIQINGIEVQNREEAVALLTSEENKNFSLLIARPELQLDEGWMDDDRNDFLDDLHMDMLEEQHHQAMQFTASVLQQKKHDEDGGTTDTATILSNQHEKDSGVGRTDESTRNDESSEQENNGDDATASSNPLAGQRKLTCSQDTLGSGDLPFSNESFISADCTDADYLGIPVDECERFRELLELKCQVKSATPYGLYYPSGPLDAGKSDPESVDKELELLNEELRSIELECLSIVRAHKMQQLKEQYRESWMLHNSGFRNYNTSIDVRRHELSDITELPEKSDKDSSSAYNTGESCRSTPLTLEISPDNSLRRAAEGISCPSSEGAVGTTEAYGPASKNLLSITEDPEVGTPTYSPSLKELDPNQPLESKERRASDGSRSPTPSQKLGSAYLPSYHHSPYKHAHIPAHAQHYQSYMQLIQQKSAVEYAQSQMSLVSMCKDLSSPTPSEPRMEWKVKIRSDGTRYITKRPVRDRLLRERALKIREERSGMTTDDDAVSEMKMGRYWSKEERKQHLVKAKEQRRRREFMMQSRLDCLKEQQAADDRKEMNILELSHKKMMKKRNKKIFDNWMTIQELLTHGTKSPDGTRVYNSFLSVTTV",
# PZRN4_HUMAN
"MGFALERFAEAVDPALECKLCGQVLEEPLCTPCGHVFCASCLLPWAVRRRRCPLQCQPLAPGELYRVLPLRSLIQKLRVQCDYRARGCGHSVRLHELEAHVEHCDFGPARRLRSRGGCASGLGGGEVPARGGCGPTPRAGRGGGARGGPPGGRWGRGRGPGPRVLAWRRREKALLAQLWALQGEVQLTARRYQEKFTQYMAHVRNFVGDLGGGHRRDGEHKPFTIVLERENDTLGFNIIGGRPNQNNQEGTSTEGIYVSKILENGPADRADGLEIHDKIMEVNGKDLSKATHEEAVEAFRNAKEPIVVQVLRRTPLSRPAYGMASEVQLMNASTQTDITFEHIMALAKLRPPTPPVPDICPFLLSDSCHSLHPMEHEFYEDNEYISSLPADADRTEDFEYEEVELCRVSSQEKLGLTVCYRTDDEEDTGIYVSEVDPNSIAAKDGRIREGDRILQINGEDVQNREEAVALLSNDECKRIVLLVARPEIQLDEGWLEDERNEFLEELNLEMLEEEHNEAMQPTANEVEQPKKQEEEEGTTDTATSSSNNHEKDSGVGRTDESLRNDESSEQENAAEDPNSTSLKSKRDLGQSQDTLGSVELQYNESLVSGEYIDSDCIGNPDEDCERFRQLLELKCKIRNHGEYDLYYSSSTIECNQGEQEGVEHELQLLNEELRNIELECQNIMQAHRLQKVTDQYGDIWTLHDGGFRNYNTSIDMQRGKLDDIMEHPEKSDKDSSSAYNTAESCRSTPLTVDRSPDSSLPRVINLTNKKNLRSTMAATQSSSGQSSKESTSTKAKTTEQGCSAESKEKVLEGSKLPDQEKAVSEHIPYLSPYHSSSYRYANIPAHARHYQSYMQLIQQKSAVEYAQSQLSLVSMCKESQKCSEPKMEWKVKIRSDGTRYITKRPVRDRILKERALKIKEERSGMTTDDDTMSEMKMGRYWSKEERKQHLVRAKEQRRRREFMMRSRLECLKESPQSGSEGKKEINIIELSHKKMMKKRNKKILDNWMTIQELMTHGAKSPDGTRVHNAFLSVTTV",
# PELI1_HUMAN
"MFSPDQENHPSKAPVKYGELIVLGYNGSLPNGDRGRRKSRFALFKRPKANGVKPSTVHIACTPQAAKAISNKDQHSISYTLSRAQTVVVEYTHDSNTDMFQIGRSTESPIDFVVTDTVPGSQSNSDTQSVQSTISRFACRIICERNPPFTARIYAAGFDSSKNIFLGEKAAKWKTSDGQMDGLTTNGVLVMHPRNGFTEDSKPGIWREISVCGNVFSLRETRSAQQRGKMVEIETNQLQDGSLIDLCGATLLWRTAEGLSHTPTVKHLEALRQEINAARPQCPVGFNTLAFPSMKRKDVVDEKQPWVYLNCGHVHGYHNWGNKEERDGKDRECPMCRSVGPYVPLWLGCEAGFYVDAGPPTHAFSPCGHVCSEKTTAYWSQIPLPHGTHTFHAACPFCAHQLAGEQGYIRLIFQGPLD",
# PELI2_HUMAN
"MFSPGQEEHCAPNKEPVKYGELVVLGYNGALPNGDRGRRKSRFALYKRPKANGVKPSTVHVISTPQASKAISCKGQHSISYTLSRNQTVVVEYTHDKDTDMFQVGRSTESPIDFVVTDTISGSQNTDEAQITQSTISRFACRIVCDRNEPYTARIFAAGFDSSKNIFLGEKAAKWKNPDGHMDGLTTNGVLVMHPRGGFTEESQPGVWREISVCGDVYTLRETRSAQQRGKLVESETNVLQDGSLIDLCGATLLWRTADGLFHTPTQKHIEALRQEINAARPQCPVGLNTLAFPSINRKEVVEEKQPWAYLSCGHVHGYHNWGHRSDTEANERECPMCRTVGPYVPLWLGCEAGFYVDAGPPTHAFTPCGHVCSEKSAKYWSQIPLPHGTHAFHAACPFCATQLVGEQNCIKLIFQGPID",
# PELI3_HUMAN
"MVLEGNPEVGSPRTSDLQHRGNKGSCVLSSPGEDAQPGEEPIKYGELIVLGCCEEGGEETEAQRGEVTGPRAHSCYNGCLASGDKGRRRSRLALSRRSHANGVKPDVMHHISTPLVSKALSNRGQHSISYTLSRSHSVIVEYTHDSDTDMFQIGRSTENMIDFVVTDTSPGGGAAEGPSAQSTISRYACRILCDRRPPYTARIYAAGFDASSNIFLGERAAKWRTPDGLMDGLTTNGVLVMHPAGGFSEDSAPGVWREISVCGNVYTLRDSRSAQQRGKLVENESNVLQDGSLIDLCGATLLWRTPAGLLRAPTLKQLEAQRQEANAARPQCPVGLSTLAFPSPARGRTAPDKQQPWVYVRCGHVHGYHGWGCRRERGPQERECPLCRLVGPYVPLWLGQEAGLCLDPGPPSHAFAPCGHVCSEKTARYWAQTPLPHGTHAFHAACPFCGAWLTGEHGCVRLIFQGPLD",
# PEX10_HUMAN
"MAPAAASPPEVIRAAQKDEYYRGGLRSAAGGALHSLAGARKWLEWRKEVELLSDVAYFGLTTLAGYQTLGEEYVSIIQVDPSRIHVPSSLRRGVLVTLHAVLPYLLDKALLPLEQELQADPDSGRPLQGSLGPGGRGCSGARRWMRHHTATLTEQQRRALLRAVFVLRQGLACLQRLHVAWFYIHGVFYHLAKRLTGITYLRVRSLPGEDLRARVSYRLLGVISLLHLVLSMGLQLYGFRQRQRARKEWRLHRGLSHRRASLEERAVSRNPLCTLCLEERRHPTATPCGHLFCWECITAWCSSKAECPLCREKFPPQKLIYLRHYR",
# PEX12_HUMAN
"MAEHGAHFTAASVADDQPSIFEVVAQDSLMTAVRPALQHVVKVLAESNPTHYGFLWRWFDEIFTLLDLLLQQHYLSRTSASFSENFYGLKRIVMGDTHKSQRLASAGLPKQQLWKSIMFLVLLPYLKVKLEKLVSSLREEDEYSIHPPSSRWKRFYRAFLAAYPFVNMAWEGWFLVQQLRYILGKAQHHSPLLRLAGVQLGRLTVQDIQALEHKPAKASMMQQPARSVSEKINSALKKAVGGVALSLSTGLSVGVFFLQFLDWWYSSENQETIKSLTALPTPPPPVHLDYNSDSPLLPKMKTVCPLCRKTRVNDTVLATSGYVFCYRCVFHYVRSHQACPITGYPTEVQHLIKLYSPEN",
# PEX2_HUMAN
"MASRKENAKSANRVLRISQLDALELNKALEQLVWSQFTQCFHGFKPGLLARFEPEVKACLWVFLWRFTIYSKNATVGQSVLNIKYKNDFSPNLRYQPPSKNQKIWYAVCTIGGRWLEERCYDLFRNHHLASFGKVKQCVNFVIGLLKLGGLINFLIFLQRGKFATLTERLLGIHSVFCKPQNICEVGFEYMNRELLWHGFAEFLIFLLPLINVQKLKAKLSSWCIPLTGAPNSDNTLATSGKECALCGEWPTMPHTIGCEHIFCYFCAKSSFLFDVYFTCPKCGTEVHSLQPLKSGIEMSEVNAL",
# PHF7_HUMAN
"MKTVKEKKECQRLRKSAKTRRVTQRKPSSGPVCWLCLREPGDPEKLGEFLQKDNISVHYFCLILSSKLPQRGQSNRGFHGFLPEDIKKEAARASRKICFVCKKKGAAINCQKDQCLRNFHLPCGQERGCLSQFFGEYKSFCDKHRPTQNIQHGHVGEESCILCCEDLSQQSVENIQSPCCSQAIYHRKCIQKYAHTSAKHFFKCPQCNNRKEFPQEMLRMGIHIPDRDAAWELEPGAFSDLYQRYQHCDAPICLYEQGRDSFEDEGRWCLILCATCGSHGTHRDCSSLRSNSKKWECEECSPAAATDYIPENSGDIPCCSSTFHPEEHFCRDNTLEENPGLSWTDWPEPSLLEKPESSRGRRSYSWRSKGVRITNSCKKSK",
# PHRF1_HUMAN
"MDDDSLDELVARSPGPDGHPQVGPADPAGDFEESSVGSSGDSGDDSDSEHGDGTDGEDEGASEEEDLEDRSGSEDSEDDGETLLEVAGTQGKLEAAGSFNSDDDAESCPICLNAFRDQAVGTPENCAHYFCLDCIVEWSKNANSCPVDRTLFKCICIRAQFGGKILRKIPVENTKASEEEEDPTFCEVCGRSDREDRLLLCDGCDAGYHMECLDPPLQEVPVDEWFCPECAAPGVVLAADAGPVSEEEVSLLLADVVPTTSRLRPRAGRTRAIARTRQSERVRATVNRNRISTARRVQHTPGRLGSSLLDEAIEAVATGLSTAVYQRPLTPRTPARRKRKTRRRKKVPGRKKTPSGPSAKSKSSATRSKKRQHRVKKRRGKKVKSEATTRSRIARTLGLRRPVHSSCIPSVLKPVEPSLGLLRADIGAASLSLFGDPYELDPFDSSEELSANPLSPLSAKRRALSRSALQSHQPVARPVSVGLSRRRLPAAVPEPDLEEEPVPDLLGSILSGQSLLMLGSSDVIIHRDGSLSAKRAAPVSFQRNSGSLSRGEEGFKGCLQPRALPSGSPAQGPSGNRPQSTGLSCQGRSRTPARTAGAPVRLDLPAAPGAVQARNLSNGSVPGFRQSHSPWFNGTNKHTLPLASAASKISSRDSKPPCRSVVPGPPLKPAPRRTDISELPRIPKIRRDDGGGRRDAAPAHGQSIEIPSACISRLTGREGTGQPGRGTRAESEASSRVPREPGVHTGSSRPPAPSSHGSLAPLGPSRGKGVGSTFESFRINIPGNMAHSSQLSSPGFCNTFRPVDDKEQRKENPSPLFSIKKTKQLRSEVYDPSDPTGSDSSAPGSSPERSGPGLLPSEITRTISINSPKAQTVQAVRCVTSYTVESIFGTEPEPPLGPSSAMSKLRGAVAAEGASDTEREEPTESQGLAARLRRPSPPEPWDEEDGASCSTFFGSEERTVTCVTVVEPEAPPSPDVLQAATHRVVELRPPSRSRSTSSSRSRKKAKRKRVSREHGRTRSGTRSESRDRSSRSASPSVGEERPRRQRSKAKSRRSSSDRSSSRERAKRKKAKDKSREHRRGPWGHSRRTSRSRSGSPGSSSYEHYESRKKKKRRSASRPRGRECSPTSSLERLCRHKHQRERSHERPDRKESVAWPRDRRKRRSRSPSSEHRAREHRRPRSREKWPQTRSHSPERKGAVREASPAPLAQGEPGREDLPTRLPALGEAHVSPEVATADKAPLQAPPVLEVAAECEPDDLDLDYGDSVEAGHVFDDFSSDAVFIQLDDMSSPPSPESTDSSPERDFPLKPALPPASLAVAAIQREVSLMHDEDPSQPPPLPEGTQEPHLLRPDAAEKAEAPSSPDVAPAGKEDSPSASGRVQEAARPEEVVSQTPLLRSRALVKRVTWNLQESESSAPAEDRAPRAPLHRPQKPREGAWDMEDVAPTGVRQVFSELPFPSHVLPEPGFPDTDPSQVYSPGLPPAPAQPSSIPPCALVSQPTVQFILQGSLPLVGCGAAQTLAPVPAALTPASEPASQATAASNSEEKTPAPRLAAEKTKKEEYMKKLHMQERAVEEVKLAIKPFYQKREVTKEEYKDILRKAVQKICHSKSGEINPVKVANLVKAYVDKYRHMRRHKKPEAGEEPPTQGAEG",
# PJA1_HUMAN
"MGQESSKPVWPNPTGGYQSNTGRRYGRRHAYVSFRPPTSQRERIASQRKTNSEVPMHRSAPSQTTKRSRSPFSTTRRSWDDSESSGTNLNIDNEDYSRYPPREYRASGSRRGMAYGHIDSYGADDSEEEGAGPVERPPVRGKTGKFKDDKLYDPEKGARSLAGPPPHFSSFSRDVREERDKLDPVPAARCSASRADFLPQSSVASQSSSEGKLATKGDSSERERREQNLPARPSRAPVSICGGGENTSKSAEEPVVRPKIRNLASPNCVKPKIFFDTDDDDDMPHSTSRWRDTANDNEGHSDGLARRGRGESSSGYPEPKYPEDKREARSDQVKPEKVPRRRRTMADPDFWTHSDDYYKYCDEDSDSDKEWIAALRRKYRSREQTLSSSGESWETLPGKEEREPPQAKVSASTGTSPGPGASASAGAGAGASAGSNGSNYLEEVREPSLQEEQASLEEGEIPWLQYHENDSSSEGDNDSGHELMQPGVFMLDGNNNLEDDSSVSEDLEVDWSLFDGFADGLGVAEAISYVDPQFLTYMALEERLAQAMETALAHLESLAVDVEVANPPASKESIDALPEILVTEDHGAVGQEMCCPICCSEYVKGEVATELPCHHYFHKPCVSIWLQKSGTCPVCRCMFPPPL",
# PJA2_HUMAN
"MSQYTEKEPAAMDQESGKAVWPKPAGGYQTITGRRYGRRHAYVSFKPCMTRHERSLGRAGDDYEVLELDDVPKENSSGSSPLDQVDSSLPSEPIFEKSETEIPTCGSALNQTTESSQSFVAVHHSEEGRDTLGSSTNLHNHSEGEYIPGACSASSVQNGIALVHTDSYDPDGKHGEDNDHLQLSAEVVEGSRYQESLGNTVFELENREAEAYTGLSPPVPSFNCEVRDEFEELDSVPLVKSSAGDTEFVHQNSQEIQRSSQDEMVSTKQQNNTSQERQTEHSPEDAACGPGHICSEQNTNDREKNHGSSPEQVVRPKVRKLISSSQVDQETGFNRHEAKQRSVQRWREALEVEESGSDDLLIKCEEYDGEHDCMFLDPPYSRVITQRETENNQMTSESGATAGRQEVDNTFWNGCGDYYQLYDKDEDSSECSDGEWSASLPHRFSGTEKDQSSSDESWETLPGKDENEPELQSDSSGPEEENQELSLQEGEQTSLEEGEIPWLQYNEVNESSSDEGNEPANEFAQPAFMLDGNNNLEDDSSVSEDLDVDWSLFDGFADGLGVAEAISYVDPQFLTYMALEERLAQAMETALAHLESLAVDVEVANPPASKESIDGLPETLVLEDHTAIGQEQCCPICCSEYIKDDIATELPCHHFFHKPCVSIWLQKSGTCPVCRRHFPPAVIEASAAPSSEPDPDAPPSNDSIAEAP",
# PLAG1_HUMAN
"MATVIPGDLSEVRDTQKVPSGKRKRGETKPRKNFPCQLCDKAFNSVEKLKVHSYSHTGERPYKCIQQDCTKAFVSKYKLQRHMATHSPEKTHKCNYCEKMFHRKDHLKNHLHTHDPNKETFKCEECGKNYNTKLGFKRHLALHAATSGDLTCKVCLQTFESTGVLLEHLKSHAGKSSGGVKEKKHQCEHCDRRFYTRKDVRRHMVVHTGRKDFLCQYCAQRFGRKDHLTRHMKKSHNQELLKVKTEPVDFLDPFTCNVSVPIKDELLPVMSLPSSELLSKPFTNTLQLNLYNTPFQSMQSSGSAHQMITTLPLGMTCPIDMDTVHPSHHLSFKYPFSSTSYAISIPEKEQPLKGEIESYLMELQGGVPSSSQDSQASSSSKLGLDPQIGSLDDGAGDLSLSKSSISISDPLNTPALDFSQLFNFIPLNGPPYNPLSVGSLGMSYSQEEAHSSVSQLPPQTQDLQDPANTIGLGSLHSLSAAFTSSLSTSTTLPRFHQAFQ",
# PLAL1_HUMAN
"MATFPCQLCGKTFLTLEKFTIHNYSHSRERPYKCVQPDCGKAFVSRYKLMRHMATHSPQKSHQCAHCEKTFNRKDHLKNHLQTHDPNKMAFGCEECGKKYNTMLGYKRHLALHAASSGDLTCGVCALELGSTEVLLDHLKAHAEEKPPSGTKEKKHQCDHCERCFYTRKDVRRHLVVHTGCKDFLCQFCAQRFGRKDHLTRHTKKTHSQELMKESLQTGDLLSTFHTISPSFQLKAAALPPFPLGASAQNGLASSLPAEVHSLTLSPPEQAAQPMQPLPESLASLHPSVSPGSPPPPLPNHKYNTTSTSYSPLASLPLKADTKGFCNISLFEDLPLQEPQSPQKLNPGFDLAKGNAGKVNLPKELPADAVNLTIPASLDLSPLLGFWQLPPPATQNTFGNSTLALGPGESLPHRLSCLGQQQQEPPLAMGTVSLGQLPLPPIPHVFSAGTGSAILPHFHHAFR",
# PML_HUMAN
"MEPAPARSPRPQQDPARPQEPTMPPPETPSEGRQPSPSPSPTERAPASEEEFQFLRCQQCQAEAKCPKLLPCLHTLCSGCLEASGMQCPICQAPWPLGADTPALDNVFFESLQRRLSVYRQIVDAQAVCTRCKESADFWCFECEQLLCAKCFEAHQWFLKHEARPLAELRNQSVREFLDGTRKTNNIFCSNPNHRTPTLTSIYCRGCSKPLCCSCALLDSSHSELKCDISAEIQQRQEELDAMTQALQEQDSAFGAVHAQMHAAVGQLGRARAETEELIRERVRQVVAHVRAQERELLEAVDARYQRDYEEMASRLGRLDAVLQRIRTGSALVQRMKCYASDQEVLDMHGFLRQALCRLRQEEPQSLQAAVRTDGFDEFKVRLQDLSSCITQGKDAAVSKKASPEAASTPRDPIDVDLPEEAERVKAQVQALGLAEAQPMAVVQSVPGAHPVPVYAFSIKGPSYGEDVSNTTTAQKRKCSQTQCPRKVIKMESEEGKEARLARSSPEQPRPSTSKAVSPPHLDGPPSPRSPVIGSEVFLPNSNHVASGAGEAEERVVVISSSEDSDAENSSSRELDDSSSESSDLQLEGPSTLRVLDENLADPQAEDRPLVFFDLKIDNETQKISQLAAVNRESKFRVVIQPEAFFSIYSKAVSLEVGLQHFLSFLSSMRRPILACYKLWGPGLPNFFRALEDINRLWEFQEAISGFLAALPLIRERVPGASSFKLKNLAQTYLARNMSERSAMAAVLAMRDLCRLLEVSPGPQLAQHVYPFSSLQCFASLQPLVQAAVLPRAEARLLALHNVSFMELLSAHRRDRQGGLKKYSRYLSLQTTTLPPAQPAFNLQALGTYFEGLLEGPALARAEGVSTPLAGRGLAERASQQS",
# PPIL2_HUMAN
"MGKRQHQKDKMYITCAEYTHFYGGKKPDLPQTNFRRLPFDHCSLSLQPFVYPVCTPDGIVFDLLNIVPWLKKYGTNPSNGEKLDGRSLIKLNFSKNSEGKYHCPVLFTVFTNNTHIVAVRTTGNVYAYEAVEQLNIKAKNFRDLLTDEPFSRQDIITLQDPTNLDKFNVSNFYHVKNNMKIIDPDEEKAKQDPSYYLKNTNAETRETLQELYKEFKGDEILAATMKAPEKKKVDKLNAAHYSTGKVSASFTSTAMVPETTHEAAAIDEDVLRYQFVKKKGYVRLHTNKGDLNLELHCDLTPKTCENFIRLCKKHYYDGTIFHRSIRNFVIQGGDPTGTGTGGESYWGKPFKDEFRPNLSHTGRGILSMANSGPNSNRSQFFITFRSCAYLDKKHTIFGRVVGGFDVLTAMENVESDPKTDRPKEEIRIDATTVFVDPYEEADAQIAQERKTQLKVAPETKVKSSQPQAGSQGPQTFRQGVGKYINPAATKRAAEEEPSTSATVPMSKKKPSRGFGDFSSW",
# PRP19_HUMAN
"MSLICSISNEVPEHPCVSPVSNHVYERRLIEKYIAENGTDPINNQPLSEEQLIDIKVAHPIRPKPPSATSIPAILKALQDEWDAVMLHSFTLRQQLQTTRQELSHALYQHDAACRVIARLTKEVTAAREALATLKPQAGLIVPQAVPSSQPSVVGAGEPMDLGELVGMTPEIIQKLQDKATVLTTERKKRGKTVPEELVKPEELSKYRQVASHVGLHSASIPGILALDLCPSDTNKILTGGADKNVVVFDKSSEQILATLKGHTKKVTSVVFHPSQDLVFSASPDATIRIWSVPNASCVQVVRAHESAVTGLSLHATGDYLLSSSDDQYWAFSDIQTGRVLTKVTDETSGCSLTCAQFHPDGLIFGTGTMDSQIKIWDLKERTNVANFPGHSGPITSIAFSENGYYLATAADDSSVKLWDLRKLKNFKTLQLDNNFEVKSLIFDQSGTYLALGGTDVQIYICKQWTEILHFTEHSGLTTGVAFGHHAKFIASTGMDRSLKFYSL",
# RAD18_HUMAN
"MDSLAESRWPPGLAVMKTIDDLLRCGICFEYFNIAMIIPQCSHNYCSLCIRKFLSYKTQCPTCCVTVTEPDLKNNRILDELVKSLNFARNHLLQFALESPAKSPASSSSKNLAVKVYTPVASRQSLKQGSRLMDNFLIREMSGSTSELLIKENKSKFSPQKEASPAAKTKETRSVEEIAPDPSEAKRPEPPSTSTLKQVTKVDCPVCGVNIPESHINKHLDSCLSREEKKESLRSSVHKRKPLPKTVYNLLSDRDLKKKLKEHGLSIQGNKQQLIKRHQEFVHMYNAQCDALHPKSAAEIVREIENIEKTRMRLEASKLNESVMVFTKDQTEKEIDEIHSKYRKKHKSEFQLLVDQARKGYKKIAGMSQKTVTITKEDESTEKLSSVCMGQEDNMTSVTNHFSQSKLDSPEELEPDREEDSSSCIDIQEVLSSSESDSCNSSSSDIIRDLLEEEEAWEASHKNDLQDTEISPRQNRRTRAAESAEIEPRNKRNRN",
# RAG1_HUMAN
"MAASFPPTLGLSSAPDEIQHPHIKFSEWKFKLFRVRSFEKTPEEAQKEKKDSFEGKPSLEQSPAVLDKADGQKPVPTQPLLKAHPKFSKKFHDNEKARGKAIHQANLRHLCRICGNSFRADEHNRRYPVHGPVDGKTLGLLRKKEKRATSWPDLIAKVFRIDVKADVDSIHPTEFCHNCWSIMHRKFSSAPCEVYFPRNVTMEWHPHTPSCDICNTARRGLKRKSLQPNLQLSKKLKTVLDQARQARQHKRRAQARISSKDVMKKIANCSKIHLSTKLLAVDFPEHFVKSISCQICEHILADPVETNCKHVFCRVCILRCLKVMGSYCPSCRYPCFPTDLESPVKSFLSVLNSLMVKCPAKECNEEVSLEKYNHHISSHKESKEIFVHINKGGRPRQHLLSLTRRAQKHRLRELKLQVKAFADKEEGGDVKSVCMTLFLLALRARNEHRQADELEAIMQGKGSGLQPAVCLAIRVNTFLSCSQYHKMYRTVKAITGRQIFQPLHALRNAEKVLLPGYHHFEWQPPLKNVSSSTDVGIIDGLSGLSSSVDDYPVDTIAKRFRYDSALVSALMDMEEDILEGMRSQDLDDYLNGPFTVVVKESCDGMGDVSEKHGSGPVVPEKAVRFSFTIMKITIAHSSQNVKVFEEAKPNSELCCKPLCLMLADESDHETLTAILSPLIAEREAMKSSELMLELGGILRTFKFIFRGTGYDEKLVREVEGLEASGSVYICTLCDATRLEASQNLVFHSITRSHAENLERYEVWRSNPYHESVEELRDRVKGVSAKPFIETVPSIDALHCDIGNAAEFYKIFQLEIGEVYKNPNASKEERKRWQATLDKHLRKKMNLKPIMRMNGNFARKLMTKETVDAVCELIPSEERHEALRELMDLYLKMKPVWRSSCPAKECPESLCQYSFNSQRFAELLSTKFKYRYEGKITNYFHKTLAHVPEIIERDGSIGAWASEGNESGNKLFRRFRKMNARQSKCYEMEDVLKHHWLYTSKYLQKFMNAHNALKTSGFTMNPQASLGDPLGIEDSLESQDSMEF",
# RAPSN_HUMAN
"MGQDQTKQQIEKGLQLYQSNQTEKALQVWTKVLEKSSDLMGRFRVLGCLVTAHSEMGRYKEMLKFAVVQIDTARELEDADFLLESYLNLARSNEKLCEFHKTISYCKTCLGLPGTRAGAQLGGQVSLSMGNAFLGLSVFQKALESFEKALRYAHNNDDAMLECRVCCSLGSFYAQVKDYEKALFFPCKAAELVNNYGKGWSLKYRAMSQYHMAVAYRLLGRLGSAMECCEESMKIALQHGDRPLQALCLLCFADIHRSRGDLETAFPRYDSAMSIMTEIGNRLGQVQALLGVAKCWVARKALDKALDAIERAQDLAEEVGNKLSQLKLHCLSESIYRSKGLQRELRAHVVRFHECVEETELYCGLCGESIGEKNSRLQALPCSHIFHLRCLQNNGTRSCPNCRRSSMKPGFV",
# RBBP6_HUMAN
"MSCVHYKFSSKLNYDTVTFDGLHISLCDLKKQIMGREKLKAADCDLQITNAQTKEEYTDDNALIPKNSSVIVRRIPIGGVKSTSKTYVISRTEPAMATTKAIDDSSASISLAQLTKTANLAEANASEEDKIKAMMSQSGHEYDPINYMKKPLGPPPPSYTCFRCGKPGHYIKNCPTNGDKNFESGPRIKKSTGIPRSFMMEVKDPNMKGAMLTNTGKYAIPTIDAEAYAIGKKEKPPFLPEEPSSSSEEDDPIPDELLCLICKDIMTDAVVIPCCGNSYCDECIRTALLESDEHTCPTCHQNDVSPDALIANKFLRQAVNNFKNETGYTKRLRKQLPPPPPPIPPPRPLIQRNLQPLMRSPISRQQDPLMIPVTSSSTHPAPSISSLTSNQSSLAPPVSGNPSSAPAPVPDITATVSISVHSEKSDGPFRDSDNKILPAAALASEHSKGTSSIAITALMEEKGYQVPVLGTPSLLGQSLLHGQLIPTTGPVRINTARPGGGRPGWEHSNKLGYLVSPPQQIRRGERSCYRSINRGRHHSERSQRTQGPSLPATPVFVPVPPPPLYPPPPHTLPLPPGVPPPQFSPQFPPGQPPPAGYSVPPPGFPPAPANLSTPWVSSGVQTAHSNTIPTTQAPPLSREEFYREQRRLKEEEKKKSKLDEFTNDFAKELMEYKKIQKERRRSFSRSKSPYSGSSYSRSSYTYSKSRSGSTRSRSYSRSFSRSHSRSYSRSPPYPRRGRGKSRNYRSRSRSHGYHRSRSRSPPYRRYHSRSRSPQAFRGQSPNKRNVPQGETEREYFNRYREVPPPYDMKAYYGRSVDFRDPFEKERYREWERKYREWYEKYYKGYAAGAQPRPSANRENFSPERFLPLNIRNSPFTRGRREDYVGGQSHRSRNIGSNYPEKLSARDGHNQKDNTKSKEKESENAPGDGKGNKHKKHRKRRKGEESEGFLNPELLETSRKSREPTGVEENKTDSLFVLPSRDDATPVRDEPMDAESITFKSVSEKDKRERDKPKAKGDKTKRKNDGSAVSKKENIVKPAKGPQEKVDGERERSPRSEPPIKKAKEETPKTDNTKSSSSSQKDEKITGTPRKAHSKSAKEHQETKPVKEEKVKKDYSKDVKSEKLTTKEEKAKKPNEKNKPLDNKGEKRKRKTEEKGVDKDFESSSMKISKLEVTEIVKPSPKRKMEPDTEKMDRTPEKDKISLSAPAKKIKLNRETGKKIGSTENISNTKEPSEKLESTSSKVKQEKVKGKVRRKVTGTEGSSSTLVDYTSTSSTGGSPVRKSEEKTDTKRTVIKTMEEYNNDNTAPAEDVIIMIQVPQSKWDKDDFESEEEDVKSTQPISSVGKPASVIKNVSTKPSNIVKYPEKESEPSEKIQKFTKDVSHEIIQHEVKSSKNSASSEKGKTKDRDYSVLEKENPEKRKNSTQPEKESNLDRLNEQGNFKSLSQSSKEARTSDKHDSTRASSNKDFTPNRDKKTDYDTREYSSSKRRDEKNELTRRKDSPSRNKDSASGQKNKPREERDLPKKGTGDSKKSNSSPSRDRKPHDHKATYDTKRPNEETKSVDKNPCKDREKHVLEARNNKESSGNKLLYILNPPETQVEKEQITGQIDKSTVKPKPQLSHSSRLSSDLTRETDEAAFEPDYNESDSESNVSVKEEESSGNISKDLKDKIVEKAKESLDTAAVVQVGISRNQSHSSPSVSPSRSHSPSGSQTRSHSSSASSAESQDSKKKKKKKEKKKHKKHKKHKKHKKHAGTEVELEKSQKHKHKKKKSKKNKDKEKEKEKDDQKVKSVTV",
# HOIL1_HUMAN
"MDEKTKKAEEMALSLTRAVAGGDEQVAMKCAIWLAEQRVPLSVQLKPEVSPTQDIRLWVSVEDAQMHTVTIWLTVRPDMTVASLKDMVFLDYGFPPVLQQWVIGQRLARDQETLHSHGVRQNGDSAYLYLLSARNTSLNPQELQRERQLRMLEDLGFKDLTLQPRGPLEPGPPKPGVPQEPGRGQPDAVPEPPPVGWQCPGCTFINKPTRPGCEMCCRARPEAYQVPASYQPDEEERARLAGEEEALRQYQQRKQQQQEGNYLQHVQLDQRSLVLNTEPAECPVCYSVLAPGEAVVLRECLHTFCRECLQGTIRNSQEAEVSCPFIDNTYSCSGKLLEREIKALLTPEDYQRFLDLGISIAENRSAFSYHCKTPDCKGWCFFEDDVNEFTCPVCFHVNCLLCKAIHEQMNCKEYQEDLALRAQNDVAARQTTEMLKVMLQQGEAMRCPQCQIVVQKKDGCDWIRCTVCHTEICWVTKGPRWGPGGPGDTSGGCRCRVNGIPCHPSCQNCH",
# RBX1_HUMAN
"MAAAMDVDTPSGTNSGAGKKRFEVKKWNAVALWAWDIVVDNCAICRNHIMDLCIECQANQASATSEECTVAWGVCNHAFHFHCISRWLKTRQVCPLDNREWEFQKYGH",
# RC3H1_HUMAN
"MPVQAPQWTDFLSCPICTQTFDETIRKPISLGCGHTVCKMCLNKLHRKACPFDQTTINTDIELLPVNSALLQLVGAQVPEQQPITLCSGVEDTKHYEEAKKCVEELALYLKPLSSARGVGLNSTTQSVLSRPMQRKLVTLVHCQLVEEEGRIRAMRAARSLGERTVTELILQHQNPQQLSSNLWAAVRARGCQFLGPAMQEEALKLVLLALEDGSALSRKVLVLFVVQRLEPRFPQASKTSIGHVVQLLYRASCFKVTKRDEDSSLMQLKEEFRTYEALRREHDSQIVQIAMEAGLRIAPDQWSSLLYGDQSHKSHMQSIIDKLQTPASFAQSVQELTIALQRTGDPANLNRLRPHLELLANIDPSPDAPPPTWEQLENGLVAVRTVVHGLVDYIQNHSKKGADQQQPPQHSKYKTYMCRDMKQRGGCPRGASCTFAHSQEELEKFRKMNKRLVPRRPLSASLGQLNEVGLPSAAILPDEGAVDLPSRKPPALPNGIVSTGNTVTQLIPRGTDPSYDSSLKPGKIDHLSSSAPGSPPDLLESVPKSISALPVNPHSIPPRGPADLPPMPVTKPLQMVPRGSQLYPAQQTDVYYQDPRGAAPPFEPAPYQQGMYYTPPPQCVSRFVRPPPSAPEPAPPYLDHYPPYLQERVVNSQYGTQPQQYPPIYPSHYDGRRVYPAPSYTREEIFRESPIPIEIPPAAVPSYVPESRERYQQIESYYPVAPHPTQIRPSYLREPPYSRLPPPPQPHPSLDELHRRRKEIMAQLEERKVISPPPFAPSPTLPPTFHPEEFLDEDLKVAGKYKGNDYSQYSPWSCDTIGSYIGTKDAKPKDVVAAGSVEMMNVESKGMRDQRLDLQRRAAETSDDDLIPFGDRPTVSRFGAISRTSKTIYQGAGPMQAMAPQGAPTKSINISDYSPYGTHGGWGASPYSPHQNIPSQGHFSERERISMSEVASHGKPLPSAEREQLRLELQQLNHQISQQTQLRGLEAVSNRLVLQREANTLAGQSQPPPPPPPKWPGMISSEQLSLELHQVEREIGKRTRELSMENQCSLDMKSKLNTSKQAENGQPEPQNKVPAEDLTLTFSDVPNGSALTQENISLLSNKTSSLNLSEDPEGGGDNNDSQRSGVTPSSAP",
# RC3H2_HUMAN
"MPVQAAQWTEFLSCPICYNEFDENVHKPISLGCSHTVCKTCLNKLHRKACPFDQTAINTDIDVLPVNFALLQLVGAQVPDHQSIKLSNLGENKHYEVAKKCVEDLALYLKPLSGGKGVASLNQSALSRPMQRKLVTLVNCQLVEEEGRVRAMRAARSLGERTVTELILQHQNPQQLSANLWAAVRARGCQFLGPAMQEEALKLVLLALEDGSALSRKVLVLFVVQRLEPRFPQASKTSIGHVVQLLYRASCFKVTKRDEDSSLMQLKEEFRSYEALRREHDAQIVHIAMEAGLRISPEQWSSLLYGDLAHKSHMQSIIDKLQSPESFAKSVQELTIVLQRTGDPANLNRLRPHLELLANIDPNPDAVSPTWEQLENAMVAVKTVVHGLVDFIQNYSRKGHETPQPQPNSKYKTSMCRDLRQQGGCPRGTNCTFAHSQEELEKYRLRNKKINATVRTFPLLNKVGVNNTVTTTAGNVISVIGSTETTGKIVPSTNGISNAENSVSQLISRSTDSTLRALETVKKVGKVGANGQNAAGPSADSVTENKIGSPPKTPVSNVAATSAGPSNVGTELNSVPQKSSPFLTRVPVYPPHSENIQYFQDPRTQIPFEVPQYPQTGYYPPPPTVPAGVAPCVPRFVRSNNVPESSLPPASMPYADHYSTFSPRDRMNSSPYQPPPPQPYGPVPPVPSGMYAPVYDSRRIWRPPMYQRDDIIRSNSLPPMDVMHSSVYQTSLRERYNSLDGYYSVACQPPSEPRTTVPLPREPCGHLKTSCEEQIRRKPDQWAQYHTQKAPLVSSTLPVATQSPTPPSPLFSVDFRADFSESVSGTKFEEDHLSHYSPWSCGTIGSCINAIDSEPKDVIANSNAVLMDLDSGDVKRRVHLFETQRRTKEEDPIIPFSDGPIISKWGAISRSSRTGYHTTDPVQATASQGSATKPISVSDYVPYVNAVDSRWSSYGNEATSSAHYVERDRFIVTDLSGHRKHSSTGDLLSLELQQAKSNSLLLQREANALAMQQKWNSLDEGRHLTLNLLSKEIELRNGELQSDYTEDATDTKPDRDIELELSALDTDEPDGQSEPIEEILDIQLGISSQNDQLLNGMAVENGHPVQQHQKEPPKQKKQSLGEDHVILEEQKTILPVTSCFSQPLPVSISNASCLPITTSVSAGNLILKTHVMSEDKNDFLKPVANGKMVNS",
# ZN363_HUMAN
"MAATAREDGASGQERGQRGCEHYDRGCLLKAPCCDKLYTCRLCHDNNEDHQLDRFKVKEVQCINCEKIQHAQQTCEECSTLFGEYYCDICHLFDKDKKQYHCENCGICRIGPKEDFFHCLKCNLCLAMNLQGRHKCIENVSRQNCPICLEDIHTSRVVAHVLPCGHLLHRTCYEEMLKEGYRCPLCMHSALDMTRYWRQLDDEVAQTPMPSEYQNMTVDILCNDCNGRSTVQFHILGMKCKICESYNTAQAGGRRISLDQQ",
# RFFL_HUMAN
"MWATCCNWFCLDGQPEEVPPPQGARMQAYSNPGYSSFPSPTGLEPSCKSCGAHFANTARKQTCLDCKKNFCMTCSSQVGNGPRLCLLCQRFRATAFQREELMKMKVKDLRDYLSLHDISTEMCREKEELVLLVLGQQPVISQEDRTRASTLSPDFPEQQAFLTQPHSSMVPPTSPNLPSSSAQATSVPPAQVQENQQANGHVSQDQEEPVYLESVARVPAEDETQSIDSEDSFVPGRRASLSDLTDLEDIEGLTVRQLKEILARNFVNYKGCCEKWELMERVTRLYKDQKGLQHLVSGAEDQNGGAVPSGLEENLCKICMDSPIDCVLLECGHMVTCTKCGKRMNECPICRQYVIRAVHVFRS",
# RFPL1_HUMAN
"MKRLSLVTTNRLSPHGNFLPLCTFPLAVDMAALFQEASSCPVCSDYLEKPMSLECGCAVCFKCINSLQKEPHGEDLLCCCCSMVSQKNKIRPSWQLERLASHIKELEPKLKKILQMNPRMRKFQVDMTLDADTANNFLLISDDLRSVRSGCITQNRQDLAERFDVSICILGSPRFTCGRHYWEVDVGTSTEWDLGVCRESVHRKGRIHLTTERGFWTVSLRDGSRLSASTVPLTFLFVDRKLQRVGIFLDMGMQNVSFFDAEGGSHVYTFRSVSAEEPLHLFFAPPSPPNGDKSVLSICPVINPGTTDAPVHPGEAK",
# RFPL2_HUMAN
"MEVAELGFPETAVSQSRICLCAVLCGHWDFADMMVIRSLSLIRLEGVEGRDPVGGGNLTNKRPSCAPSPQDLSAQWKQLEDRGASSRRVDMAALFQEASSCPVCSDYLEKPMSLECGCAVCLKCINSLQKEPHGEDLLCCCSSMVSRKNKIRRNRQLERLASHIKELEPKLKKILQMNPRMRKFQVDMTLDANTANNFLLISDDLRSVRSGRIRQNRQDLAERFDVSVCILGSPRFTCGRHCWEVDVGTSTEWDLGVCRESVHRKGRIQLTTELGFWTVSLRDGGRLSATTVPLTFLFVDRKLQRVGIFLDMGMQNVSFFDAESGSHVYTFRSVSAEEPLRPFLAPSVPPNGDQGVLSICPLMNSGTTDAPVRPGEAK",
# RFPL3_HUMAN
"MKRLSLVTTNRLSPQGNFLPLCTFPLAVDMAALFQEASSCPVCSDYLEKPMSLECGCTVCLKCINSLQKEPHGEDLLCCCCSMVSQRNKIRPNRQLERLVSHIKELEPKLKKILQMNPRMRKFQVDMTLDADTANNFLLISDDLRSVRSGLITQNRQDLAERFDVSVCILGSPRFTCGRHYWEVDVGTSTEWDLGVCRESVHCKGKIQLTTELGFWTVSLRDGSRLSASTVPLTFLLVDRKLQRVGIFLDMGMQNVSFFDAESGSHVYTFRSVSAEEPLRPFLAPSIPPNGDQGVLSICPLMNSGTTDAPVRPGEAK",
# RFPLA_HUMAN
"MAEHFKQIIRCPVCLKDLEEAVQLKCGYACCLQCLNSLQKEPDGEGLLCRFCSVVSQKDDIKPKYKLRALVSIIKELEPKLKSVLTMNPRMRKFQVDMTFDVDTANNYLIISEDLRSFRSGDLSQNRKEQAERFDTALCVLGTPRFTSGRHYWEVDVGTSQVWDVGVCKESVNRQGKIVLSSEHGFLTVGCREGKVFAASTVPMTPLWVSPQLHRVGIFLDVGMRSIAFYNVSDGCHIYTFIEIPVCEPWRPFFAHKRGSQDDQSILSICSVINPSAASAPVSSEGK",
# RFAL1_HUMAN
"MAEHFKQIIRCPVCLKDLEEAVQLKCGYACCLQCLNSLQKEPDGEGLLCRFCSVVSQKDDIKPKYKLRALVSIIKELEPKLKSVLTMNPRMRKFQVDMTFDVDTANNYLIISEDLRSFRSGDLSQNRKEQAERFDTALCVLGTPRFTSGRHYWEVDVGTSQVWDVGVCKESVNRQGKIELSSEHGFLTVGCREGKVFAASTVPMTPLWVSPQLHRVGIFLDVGMRSIAFYNVSDGCHINTFIEIPVCEPWRPFFAHKRGSQDDQSILSICSVINPSTASAPVSSEGK",
# RFPLB_HUMAN
"MAKRLQAELSCPVCLDFFSCSISLSCTHVFCFDCIQRYILENHDFRAMCPLCRDVVKVPALEEWQVSVLTLMTKQHNSRLEQSLHVREELRHFREDVTLDAATASSLLVFSNDLRSAQCKKIHHDLTKDPRLACVLGTPCFSSGQHYWEVEVGEVKSWSLGVCKEPADRKSNDLFPEHGFWISMKAGAIHANTHLERIPASPRLRRVGIFLDADLEEIQFFDVDNNVLIYTHDGFFSLELLCPFFCLELLGEGESGNVLTICP",
# COP1_HUMAN
"MSGSRQAGSGSAGTSPGSSAASSVTSASSSLSSSPSPPSVAVSAAALVSGGVAQAAGSGGLGGPVRPVLVAPAVSGSGGGAVSTGLSRHSCAARPSAGVGGSSSSLGSGSRKRPLLAPLCNGLINSYEDKSNDFVCPICFDMIEEAYMTKCGHSFCYKCIHQSLEDNNRCPKCNYVVDNIDHLYPNFLVNELILKQKQRFEEKRFKLDHSVSSTNGHRWQIFQDWLGTDQDNLDLANVNLMLELLVQKKKQLEAESHAAQLQILMEFLKVARRNKREQLEQIQKELSVLEEDIKRVEEMSGLYSPVSEDSTVPQFEAPSPSHSSIIDSTEYSQPPGFSGSSQTKKQPWYNSTLASRRKRLTAHFEDLEQCYFSTRMSRISDDSRTASQLDEFQECLSKFTRYNSVRPLATLSYASDLYNGSSIVSSIEFDRDCDYFAIAGVTKKIKVYEYDTVIQDAVDIHYPENEMTCNSKISCISWSSYHKNLLASSDYEGTVILWDGFTGQRSKVYQEHEKRCWSVDFNLMDPKLLASGSDDAKVKLWSTNLDNSVASIEAKANVCCVKFSPSSRYHLAFGCADHCVHYYDLRNTKQPIMVFKGHRKAVSYAKFVSGEEIVSASTDSQLKLWNVGKPYCLRSFKGHINEKNFVGLASNGDYIACGSENNSLYLYYKGLSKTLLTFKFDTVKSVLDKDRKEDDTNEFVSAVCWRALPDGESNVLIAANSQGTIKVLELV",
# RFWD3_HUMAN
"MAHEAMEYDVQVQLNHAEQQPAPAGMASSQGGPALLQPVPADVVSSQGVPSILQPAPAEVISSQATPPLLQPAPQLSVDLTEVEVLGEDTVENINPRTSEQHRQGSDGNHTIPASSLHSMTNFISGLQRLHGMLEFLRPSSSNHSVGPMRTRRRVSASRRARAGGSQRTDSARLRAPLDAYFQVSRTQPDLPATTYDSETRNPVSEELQVSSSSDSDSDSSAEYGGVVDQAEESGAVILEEQLAGVSAEQEVTCIDGGKTLPKQPSPQKSEPLLPSASMDEEEGDTCTICLEQWTNAGDHRLSALRCGHLFGYRCISTWLKGQVRKCPQCNKKARHSDIVVLYARTLRALDTSEQERMKSSLLKEQMLRKQAELESAQCRLQLQVLTDKCTRLQRRVQDLQKLTSHQSQNLQQPRGSQAWVLSCSPSSQGQHKHKYHFQKTFTVSQAGNCRIMAYCDALSCLVISQPSPQASFLPGFGVKMLSTANMKSSQYIPMHGKQIRGLAFSSYLRGLLLSASLDNTIKLTSLETNTVVQTYNAGRPVWSCCWCLDEANYIYAGLANGSILVYDVRNTSSHVQELVAQKARCPLVSLSYMPRAASAAFPYGGVLAGTLEDASFWEQKMDFSHWPHVLPLEPGGCIDFQTENSSRHCLVTYRPDKNHTTIRSVLMEMSYRLDDTGNPICSCQPVHTFFGGPTCKLLTKNAIFQSPENDGNILVCTGDEAANSALLWDAASGSLLQDLQTDQPVLDICPFEVNRNSYLATLTEKMVHIYKWE",
# RING1_HUMAN
"MTTPANAQNASKTWELSLYELHRTPQEAIMDGTEIAVSPRSLHSELMCPICLDMLKNTMTTKECLHRFCSDCIVTALRSGNKECPTCRKKLVSKRSLRPDPNFDALISKIYPSREEYEAHQDRVLIRLSRLHNQQALSSSIEEGLRMQAMHRAQRVRRPIPGSDQTTTMSGGEGEPGEGEGDGEDVSSDSAPDSAPGPAPKRPRGGGAGGSSVGTGGGGTGGVGGGAGSEDSGDRGGTLGGGTLGPPSPPGAPSPPEPGGEIELVFRPHPLLVEKGEYCQTRYVKTTGNATVDHLSKYLALRIALERRQQQEAGEPGGPGGGASDTGGPDGCGGEGGGAGGGDGPEEPALPSLEGVSEKQYTIYIAPGGGAFTTLNGSLTLELVNEKFWKVSRPLELCYAPTKDPK",
# RLF_HUMAN
"MADGKGDAAAVAGAGAEAPAVAGAGDGVETESMVRGHRPVSPAPGASGLRPCLWQLETELREQEVSEVSSLNYCRSFCQTLLQYASNKNASEHIVYLLEVYRLAIQSFASARPYLTTECEDVLLVLGRLVLSCFELLLSVSESELPCEVWLPFLQSLQESHDALLEFGNNNLQILVHVTKEGVWKNPVLLKILSQQPVETEEVNKLIAQEGPSFLQMRIKHLLKSNCIPQATALSKLCAESKEISNVSSFQQAYITCLCSMLPNEDAIKEIAKVDCKEVLDIICNLESEGQDNTAFVLCTTYLTQQLQTASVYCSWELTLFWSKLQRRIDPSLDTFLERCRQFGVIAKTQQHLFCLIRVIQTEAQDAGLGVSILLCVRALQLRSSEDEEMKASVCKTIACLLPEDLEVRRACQLTEFLIEPSLDGFNMLEELYLQPDQKFDEENAPVPNSLRCELLLALKAHWPFDPEFWDWKTLKRHCHQLLGQEASDSDDDLSGYEMSINDTDVLESFLSDYDEGKEDKQYRRRDLTDQHKEKRDKKPIGSSERYQRWLQYKFFCLLCKRECIEARILHHSKMHMEDGIYTCPVCIKKFKRKEMFVPHVMEHVKMPPSRRDRSKKKLLLKGSQKGICPKSPSAIPEQNHSLNDQAKGESHEYVTFSKLEDCHLQDRDLYPCPGTDCSRVFKQFKYLSVHLKAEHQNNDENAKHYLDMKNRREKCTYCRRHFMSAFHLREHEQVHCGPQPYMCVSIDCYARFGSVNELLNHKQKHDDLRYKCELNGCNIVFSDLGQLYHHEAQHFRDASYTCNFLGCKKFYYSKIEYQNHLSMHNVENSNGDIKKSVKLEESATGEKQDCINQPHLLNQTDKSHLPEDLFCAESANSQIDTETAENLKENSDSNSSDQLSHSSSASMNEELIDTLDHSETMQDVLLSNEKVFGPSSLKEKCSSMAVCFDGTKFTCGFDGCGSTYKNARGMQKHLRKVHPYHFKPKKIKTKDLFPSLGNEHNQTTEKLDAEPKPCSDTNSDSPDEGLDHNIHIKCKREHQGYSSESSICASKRPCTEDTMLELLLRLKHLSLKNSITHGSFSGSLQGYPSSGAKSLQSVSSISDLNFQNQDENMPSQYLAQLAAKPFFCELQGCKYEFVTREALLMHYLKKHNYSKEKVLQLTMFQHRYSPFQCHICQRSFTRKTHLRIHYKNKHQIGSDRATHKLLDNEKCDHEGPCSVDRLKGDCSAELGGDPSSNSEKPHCHPKKDECSSETDLESSCEETESKTSDISSPIGSHREEQEGREGRGSRRTVAKGNLCYILNKYHKPFHCIHKTCNSSFTNLKGLIRHYRTVHQYNKEQLCLEKDKARTKRELVKCKKIFACKYKECNKRFLCSKALAKHCSDSHNLDHIEEPKVLSEAGSAARFSCNQPQCPAVFYTFNKLKHHLMEQHNIEGEIHSDYEIHCDLNGCGQIFTHRSNYSQHVYYRHKDYYDDLFRSQKVANERLLRSEKVCQTADTQGHEHQTTRRSFNAKSKKCGLIKEKKAPISFKTRAEALHMCVEHSEHTQYPCMVQGCLSVVKLESSIVRHYKRTHQMSSAYLEQQMENLVVCVKYGTKIKEEPPSEADPCIKKEENRSCESERTEHSHSPGDSSAPIQNTDCCHSSERDGGQKGCIESSSVFDADTLLYRGTLKCNHSSKTTSLEQCNIVQPPPPCKIENSIPNPNGTESGTYFTSFQLPLPRIKESETRQHSSGQENTVKNPTHVPKENFRKHSQPRSFDLKTYKPMGFESSFLKFIQESEEKEDDFDDWEPSEHLTLSNSSQSSNDLTGNVVANNMVNDSEPEVDIPHSSSDSTIHENLTAIPPLIVAETTTVPSLENLRVVLDKALTDCGELALKQLHYLRPVVVLERSKFSTPILDLFPTKKTDELCVGSS",
# RNF12_HUMAN
"MENSDSNDKGSGDQSAAQRRSQMDRLDREEAFYQFVNNLSEEDYRLMRDNNLLGTPGESTEEELLRRLQQIKEGPPPQNSDENRGGDSSDDVSNGDSIIDWLNSVRQTGNTTRSGQRGNQSWRAVSRTNPNSGDFRFSLEINVNRNNGSQNSENENEPSARRSSGENVENNSQRQVENPRSESTSARPSRSERNSTEALTEVPPTRGQRRARSRSPDHRRTRARAERSRSPLHPMSEIPRRSHHSISSQTFEHPLVNETEGSSRTRHHVTLRQQISGPELLSRGLFAASGTRNASQGAGSSDTAASGESTGSGQRPPTIVLDLQVRRVRPGEYRQRDSIASRTRSRSQTPNNTVTYESERGGFRRTFSRSERAGVRTYVSTIRIPIRRILNTGLSETTSVAIQTMLRQIMTGFGELSYFMYSDSDSEPTGSVSNRNMERAESRSGRGGSGGGSSSGSSSSSSSSSSSSSSSSSSSSPSSSSGGESSETSSDLFEGSNEGSSSSGSSGARREGRHRAPVTFDESGSLPFLSLAQFFLLNEDDDDQPRGLTKEQIDNLAMRSFGENDALKTCSVCITEYTEGNKLRKLPCSHEYHVHCIDRWLSENSTCPICRRAVLASGNRESVV",
# RMD5A_HUMAN
"MDQCVTVERELEKVLHKFSGYGQLCERGLEELIDYTGGLKHEILQSHGQDAELSGTLSLVLTQCCKRIKDTVQKLASDHKDIHSSVSRVGKAIDKNFDSDISSVGIDGCWQADSQRLLNEVMVEHFFRQGMLDVAEELCQESGLSVDPSQKEPFVELNRILEALKVRVLRPALEWAVSNREMLIAQNSSLEFKLHRLYFISLLMGGTTNQREALQYAKNFQPFALNHQKDIQVLMGSLVYLRQGIENSPYVHLLDANQWADICDIFTRDACALLGLSVESPLSVSFSAGCVALPALINIKAVIEQRQCTGVWNQKDELPIEVDLGKKCWYHSIFACPILRQQTTDNNPPMKLVCGHIISRDALNKMFNGSKLKCPYCPMEQSPGDAKQIFF",
# RMD5B_HUMAN
"MEQCACVERELDKVLQKFLTYGQHCERSLEELLHYVGQLRAELASAALQGTPLSATLSLVMSQCCRKIKDTVQKLASDHKDIHSSVSRVGKAIDRNFDSEICGVVSDAVWDAREQQQQILQMAIVEHLYQQGMLSVAEELCQESTLNVDLDFKQPFLELNRILEALHEQDLGPALEWAVSHRQRLLELNSSLEFKLHRLHFIRLLAGGPAKQLEALSYARHFQPFARLHQREIQVMMGSLVYLRLGLEKSPYCHLLDSSHWAEICETFTRDACSLLGLSVESPLSVSFASGCVALPVLMNIKAVIEQRQCTGVWNHKDELPIEIELGMKCWYHSVFACPILRQQTSDSNPPIKLICGHVISRDALNKLINGGKLKCPYCPMEQNPADGKRIIF",
# RNF10_HUMAN
"MPLSSPNAAATASDMDKNSGSNSSSASSGSSKGQQPPRSASAGPAGESKPKSDGKNSSGSKRYNRKRELSYPKNESFNNQSRRSSSQKSKTFNKMPPQRGGGSSKLFSSSFNGGRRDEVAEAQRAEFSPAQFSGPKKINLNHLLNFTFEPRGQTGHFEGSGHGSWGKRNKWGHKPFNKELFLQANCQFVVSEDQDYTAHFADPDTLVNWDFVEQVRICSHEVPSCPICLYPPTAAKITRCGHIFCWACILHYLSLSEKTWSKCPICYSSVHKKDLKSVVATESHQYVVGDTITMQLMKREKGVLVALPKSKWMNVDHPIHLGDEQHSQYSKLLLASKEQVLHRVVLEEKVALEQQLAEEKHTPESCFIEAAIQELKTREEALSGLAGSRREVTGVVAALEQLVLMAPLAKESVFQPRKGVLEYLSAFDEETTEVCSLDTPSRPLALPLVEEEEAVSEPEPEGLPEACDDLELADDNLKEGTICTESSQQEPITKSGFTRLSSSPCYYFYQAEDGQHMFLHPVNVRCLVREYGSLERSPEKISATVVEIAGYSMSEDVRQRHRYLSHLPLTCEFSICELALQPPVVSKETLEMFSDDIEKRKRQRQKKAREERRRERRIEIEENKKQGKYPEVHIPLENLQQFPAFNSYTCSSDSALGPTSTEGHGALSISPLSRSPGSHADFLLTPLSPTASQGSPSFCVGSLEEDSPFPSFAQMLRVGKAKADVWPKTAPKKDENSLVPPAPVDSDGESDNSDRVPVPSFQNSFSQAIEAAFMKLDTPATSDPLSEEKGGKKRKKQKQKLLFSTSVVHTK",
# RN103_HUMAN
"MWLKLFFLLLYFLVLFVLARFFEAIVWYETGIFATQLVDPVALSFKKLKTILECRGLGYSGLPEKKDVRELVEKSGDLMEGELYSALKEEEASESVSSTNFSGEMHFYELVEDTKDGIWLVQVIANDRSPLVGKIHWEKMVKKVSRFGIRTGTFNCSSDPRYCRRRGWVRSTLIMSVPQTSTSKGKVMLKEYSGRKIEVEHIFKWITAHAASRIKTIYNAEHLKEEWNKSDQYWLKIYLFANLDQPPAFFSALSIKFTGRVEFIFVNVENWDNKSYMTDIGIYNMPSYILRTPEGIYRYGNHTGEFISLQAMDSFLRSLQPEVNDLFVLSLVLVNLMAWMDLFITQGATIKRFVVLISTLGTYNSLLIISWLPVLGFLQLPYLDSFYEYSLKLLRYSNTTTLASWVRADWMFYSSHPALFLSTYLGHGLLIDYFEKKRRRNNNNDEVNANNLEWLSSLWDWYTSYLFHPIASFQNFPVESDWDEDPDLFLERLAFPDLWLHPLIPTDYIKNLPMWRFKCLGVQSEEEMSEGSQDTENDSESENTDTLSSEKEVFEDKQSVLHNSPGTASHCDAEACSCANKYCQTSPCERKGRSYGSYNTNEDMEPDWLTWPADMLHCTECVVCLENFENGCLLMGLPCGHVFHQNCIVMWLAGGRHCCPVCRWPSYKKKQPYAQHQPLSNDVPS",
# RNF11_HUMAN
"MGNCLKSPTSDDISLLHESQSDRASFGEGTEPDQEPPPPYQEQVPVPVYHPTPSQTRLATQLTEEEQIRIAQRIGLIQHLPKGVYDPGRDGSEKKIRECVICMMDFVYGDPIRFLPCMHIYHLDCIDDWLMRSFTCPSCMEPVDAALLSSYETN",
# RN111_HUMAN
"MSQWTPEYNELYTLKVDMKSEIPSDAPKTQESLKGILLHPEPIGAAKSFPAGVEMINSKVGNEFSHLCDDSQKQEKEMNGNQQEQEKSLVVRKKRKSQQAGPSYVQNCVKENQGILGLRQHLGTPSDEDNDSSFSDCLSSPSSSLHFGDSDTVTSDEDKEVSVRHSQTILNAKSRSHSARSHKWPRTETESVSGLLMKRPCLHGSSLRRLPCRKRFVKNNSSQRTQKQKERILMQRKKREVLARRKYALLPSSSSSSENDLSSESSSSSSTEGEEDLFVSASENHQNNPAVPSGSIDEDVVVIEASSTPQVTANEEINVTSTDSEVEIVTVGESYRSRSTLGHSRSHWSQGSSSHASRPQEPRNRSRISTVIQPLRQNAAEVVDLTVDEDEPTVVPTTSARMESQATSASINNSNPSTSEQASDTASAVTSSQPSTVSETSATLTSNSTTGTSIGDDSRRTTSSAVTETGPPAMPRLPSCCPQHSPCGGSSQNHHALGHPHTSCFQQHGHHFQHHHHHHHTPHPAVPVSPSFSDPACPVERPPQVQAPCGANSSSGTSYHEQQALPVDLSNSGIRSHGSGSFHGASAFDPCCPVSSSRAAIFGHQAAAAAPSQPLSSIDGYGSSMVAQPQPQPPPQPSLSSCRHYMPPPYASLTRPLHHQASACPHSHGNPPPQTQPPPQVDYVIPHPVHAFHSQISSHATSHPVAPPPPTHLASTAAPIPQHLPPTHQPISHHIPATAPPAQRLHPHEVMQRMEVQRRRMMQHPTRAHERPPPHPHRMHPNYGHGHHIHVPQTMSSHPRQAPERSAWELGIEAGVTAATYTPGALHPHLAHYHAPPRLHHLQLGALPLMVPDMAGYPHIRYISSGLDGTSFRGPFRGNFEELIHLEERLGNVNRGASQGTIERCTYPHKYKKVTTDWFSQRKLHCKQDGEEGTEEDTEEKCTICLSILEEGEDVRRLPCMHLFHQVCVDQWLITNKKCPICRVDIEAQLPSES",
# RN112_HUMAN
"MPRPALSVTSFCHRLGKRERKQSFMGNSGNSWSHTPFPKLELGLGPQPMAPRELPTCSICLERLRDPISLDCGHDFCIRCFSTHRLPGCEPPCCPECRKICKQKRGLRSLGEKMKLLPQRPLPPALQETCPVRAEPLLLVRINASGGLILRMGAINRCLKHPLARDTPVCLLAVLGEQHSGKSFLLNHLLQGLPGLESGEGGRPRGGEASLQGCRWGANGLARGIWMWSHPFLLGKEGKKVAVFLVDTGDAMSPELSRETRIKLCALTTMLSSYQILSTSQELKDTDLDYLEMFVHVAEVMGKHYGMVPIQHLDLLVRDSSHPNKAGQGHVGNIFQRLSGRYPKVQELLQGKRARCCLLPAPGRRRMNQGHASPGDTDDDFRHLLGAYVSDVLSAAPQHAKSRCQGYWNEGRAVARGDRRLLTGQQLAQEIKNLSGWMGRTGPGFTSPDEMAAQLHDLRKVEAAKREFEEYVRQQDVATKRIFSALRVLPDTMRNLLSTQKDAILARHGVALLCKGRDQTLEALEAELQATAKAFMDSYTMRFCGHLAAVGGAVGAGLMGLAGGVVGAGMAAAALAAEAGMVAAGAAVGATGAAVVGGGVGAGLAATVGCMEKEEDERLLEGDREPLLQEE",
# R113A_HUMAN
"MAEQLSPGKAVDQVCTFLFKKPGRKGAAGRRKRPACDPEPGESGSSSDEGCTVVRPEKKRVTHNPMIQKTRDSGKQKAAYGDLSSEEEEENEPESLGVVYKSTRSAKPVGPEDMGATAVYELDTEKERDAQAIFERSQKIQEELRGKEDDKIYRGINNYQKYMKPKDTSMGNASSGMVRKGPIRAPEHLRATVRWDYQPDICKDYKETGFCGFGDSCKFLHDRSDYKHGWQIERELDEGRYGVYEDENYEVGSDDEEIPFKCFICRQSFQNPVVTKCRHYFCESCALQHFRTTPRCYVCDQQTNGVFNPAKELIAKLEKHRATGEGGASDLPEDPDEDAIPIT",
# R113B_HUMAN
"MAAPPSPGRTADQADQVCTFLFKKPGRKGAAGLRKRPACDPEHGESSSSGDEGDTVAQPPRVAPRPRGLHSWQKAAHGDRRGEEAAPESLDVVYRSTRSAKPVGPEDMGATADFEQDTEKEHHTPTILKCSQRVQEALRGREHDHIYRGIHSYLRYLKPKDTSMGNSSSGMARKGPIRAPGHLRATVRWDYQPDICKDYKETGFCGFGDSCKFLHDRSDYKLGWEIERELEEGRYCICEDENHEVGSEEEEIPFRCFICRQAFQNPVVTKCRHYFCESCALEHFRATPRCYICDQPTGGIFNPAKELMAKLQKLQAAEGKKR",
# RN114_HUMAN
"MAAQQRDCGGAAQLAGPAAEADPLGRFTCPVCLEVYEKPVQVPCGHVFCSACLQECLKPKKPVCGVCRSALAPGVRAVELERQIESTETSCHGCRKNFFLSKIRSHVATCSKYQNYIMEGVKATIKDASLQPRNVPNRYTFPCPYCPEKNFDQEGLVEHCKLFHSTDTKSVVCPICASMPWGDPNYRSANFREHIQRRHRFSYDTFVDYDVDEEDMMNQVLQRSIIDQ",
# RN115_HUMAN
"MAEASAAGADSGAAVAAHRFFCHFCKGEVSPKLPEYICPRCESGFIEEVTDDSSFLGGGGSRIDNTTTTHFAELWGHLDHTMFFQDFRPFLSSSPLDQDNRANERGHQTHTDFWGARPPRLPLGRRYRSRGSSRPDRSPAIEGILQHIFAGFFANSAIPGSPHPFSWSGMLHSNPGDYAWGQTGLDAIVTQLLGQLENTGPPPADKEKITSLPTVTVTQEQVDMGLECPVCKEDYTVEEEVRQLPCNHFFHSSCIVPWLELHDTCPVCRKSLNGEDSTRQSQSTEASASNRFSNDSQLHDRWTF",
# RN121_HUMAN
"MAAVVEVEVGGGAAGERELDEVDMSDLSPEEQWRVEHARMHAKHRGHEAMHAEMVLILIATLVVAQLLLVQWKQRHPRSYNMVTLFQMWVVPLYFTVKLHWWRFLVIWILFSAVTAFVTFRATRKPLVQTTPRLVYKWFLLIYKISYATGIVGYMAVMFTLFGLNLLFKIKPEDAMDFGISLLFYGLYYGVLERDFAEMCADYMASTIGFYSESGMPTKHLSDSVCAVCGQQIFVDVSEEGIIENTYRLSCNHVFHEFCIRGWCIVGKKQTCPYCKEKVDLKRMFSNPWERPHVMYGQLLDWLRYLVAWQPVIIGVVQGINYILGLE",
# RN122_HUMAN
"MHPFQWCNGCFCGLGLVSTNKSCSMPPISFQDLPLNIYMVIFGTGIFVFMLSLIFCCYFISKLRNQAQSERYGYKEVVLKGDAKKLQLYGQTCAVCLEDFKGKDELGVLPCQHAFHRKCLVKWLEVRCVCPMCNKPIASPSEATQNIGILLDELV",
# RN123_HUMAN
"MASKGAGMSFSRKSYRLTSDAEKSRVTGIVQEKLLNDYLNRIFSSSEHAPPAATSRKPLNFQNLPEHLDQLLQVDNEEEESQGQVEGRLGPSTVVLDHTGGFEGLLLVDDDLLGVIGHSNFGTIRSTTCVYKGKWLYEVLISSQGLMQIGWCTISCRFNQEEGVGDTHNSYAYDGNRVRKWNVTTTNYGKAWAAGDIVSCLIDLDDGTLSFCLNGVSLGTAFENLSRGLGMAYFPAISLSFKESVAFNFGSRPLRYPVAGYRPLQDPPSADLVRAQRLLGCFRAVLSVELDPVEGRLLDKESSKWRLRGQPTVLLTLAHIFHHFAPLLRKVYLVEAVLMSFLLGIVEKGTPTQAQSVVHQVLDLLWLFMEDYEVQDCLKQLMMSLLRLYRFSPIVPDLGLQIHYLRLTIAILRHEKSRKFLLSNVLFDVLRSVVFFYIKSPLRVEEAGLQELIPTTWWPHCSSREGKESTEMKEETAEERLRRRAYERGCQRLRKRIEVVEELQVQILKLLLDNKDDNGGEASRYIFLTKFRKFLQENASGRGNMPMLCPPEYMVCFLHRLISALRYYWDEYKASNPHASFSEEAYIPPQVFYNGKVDYFDLQRLGGLLSHLRKTLKDDLASKANIVIDPLELQSTAMDDLDEDEEPAPAMAQRPMQALAVGGPLPLPRPGWLSSPTLGRANRFLSTAAVSLMTPRRPLSTSEKVKVRTLSVEQRTREDIEGSHWNEGLLLGRPPEEPEQPLTENSLLEVLDGAVMMYNLSVHQQLGKMVGVSDDVNEYAMALRDTEDKLRRCPKRRKDILAELTKSQKVFSEKLDHLSRRLAWVHATVYSQEKMLDIYWLLRVCLRTIEHGDRTGSLFAFMPEFYLSVAINSYSALKNYFGPVHSMEELPGYEETLTRLAAILAKHFADARIVGTDIRDSLMQALASYVCYPHSLRAVERIPEEQRIAMVRNLLAPYEQRPWAQTNWILVRLWRGCGFGYRYTRLPHLLKTKLEDANLPSLQKPCPSTLLQQHMADLLQQGPDVAPSFLNSVLNQLNWAFSEFIGMIQEIQQAAERLERNFVDSRQLKVCATCFDLSVSLLRVLEMTITLVPEIFLDWTRPTSEMLLRRLAQLLNQVLNRVTAERNLFDRVVTLRLPGLESVDHYPILVAVTGILVQLLVRGPASEREQATSVLLADPCFQLRSICYLLGQPEPPAPGTALPAPDRKRFSLQSYADYISADELAQVEQMLAHLTSASAQAAAASLPTSEEDLCPICYAHPISAVFQPCGHKSCKACINQHLMNNKDCFFCKTTIVSVEDWEKGANTSTTSSAA",
# RN125_HUMAN
"MGSVLSTDSGKSAPASATARALERRRDPELPVTSFDCAVCLEVLHQPVRTRCGHVFCRSCIATSLKNNKWTCPYCRAYLPSEGVPATDVAKRMKSEYKNCAECDTLVCLSEMRAHIRTCQKYIDKYGPLQELEETAARCVCPFCQRELYEDSLLDHCITHHRSERRPVFCPLCRLIPDENPSSFSGSLIRHLQVSHTLFYDDFIDFNIIEEALIRRVLDRSLLEYVNHSNTT",
# RN126_HUMAN
"MAEASPHPGRYFCHCCSVEIVPRLPDYICPRCESGFIEELPEETRSTENGSAPSTAPTDQSRPPLEHVDQHLFTLPQGYGQFAFGIFDDSFEIPTFPPGAQADDGRDPESRRERDHPSRHRYGARQPRARLTTRRATGRHEGVPTLEGIIQQLVNGIITPATIPSLGPWGVLHSNPMDYAWGANGLDAIITQLLNQFENTGPPPADKEKIQALPTVPVTEEHVGSGLECPVCKDDYALGERVRQLPCNHLFHDGCIVPWLEQHDSCPVCRKSLTGQNTATNPPGLTGVSFSSSSSSSSSSSPSNENATSNS",
# RN128_HUMAN
"MGPPPGAGVSCRGGCGFSRLLAWCFLLALSPQAPGSRGAEAVWTAYLNVSWRVPHTGVNRTVWELSEEGVYGQDSPLEPVAGVLVPPDGPGALNACNPHTNFTVPTVWGSTVQVSWLALIQRGGGCTFADKIHLAYERGASGAVIFNFPGTRNEVIPMSHPGAVDIVAIMIGNLKGTKILQSIQRGIQVTMVIEVGKKHGPWVNHYSIFFVSVSFFIITAATVGYFIFYSARRLRNARAQSRKQRQLKADAKKAIGRLQLRTLKQGDKEIGPDGDSCAVCIELYKPNDLVRILTCNHIFHKTCVDPWLLEHRTCPMCKCDILKALGIEVDVEDGSVSLQVPVSNEISNSASSHEEDNRSETASSGYASVQGTDEPPLEEHVQSTNESLQLVNHEANSVAVDVIPHVDNPTFEEDETPNQETAVREIKS",
# MENB_BACSU
"MAEWKTKRTYDEILYETYNGIAKITINRPEVHNAFTPKTVAEMIDAFADARDDQNVGVIVLAGAGDKAFCSGGDQKVRGHGGYVGDDQIPRLNVLDLQRLIRVIPKPVVAMVSGYAIGGGHVLHIVCDLTIAADNAIFGQTGPKVGSFDAGYGSGYLARIVGHKKAREIWYLCRQYNAQEALDMGLVNTVVPLEQLEEETIKWCEEMLEKSPTALRFLKAAFNADTDGLAGIQQFAGDATLLYYTTDEAKEGRDSFKEKRKPDFGQFPRFP",
# GOLI_HUMAN
"MSCAGRAGPARLAALALLTCSLWPARADNASQEYYTALINVTVQEPGRGAPLTFRIDRGRYGLDSPKAEVRGQVLAPLPLHGVADHLGCDPQTRFFVPPNIKQWIALLQRGNCTFKEKISRAAFHNAVAVVIYNNKSKEEPVTMTHPGTGDIIAVMITELRGKDILSYLEKNISVQMTIAVGTRMPPKNFSRGSLVFVSISFIVLMIISSAWLIFYFIQKIRYTNARDRNQRRLGDAAKKAISKLTTRTVKKGDKETDPDFDHCAVCIESYKQNDVVRILPCKHVFHKSCVDPWLSEHCTCPMCKLNILKALGIVPNLPCTDNVAFDMERLTRTQAVNRRSALGDLAGDNSLGLEPLRTSGISPLPQDGELTPRTGEINIAVTKEWFIIASFGLLSALTLCYMIIRATASLNANEVEWF",
# RN133_HUMAN
"MHLLKVGTWRNNTASSWLMKFSVLWLVSQNCCRASVVWMAYMNISFHVGNHVLSELGETGVFGRSSTLKRVAGVIVPPEGKIQNACNPNTIFSRSKYSETWLALIERGGCTFTQKIKVATEKGASGVIIYNVPGTGNQVFPMFHQAFEDVVVVMIGNLKGTEIFHLIKKGVLITAVVEVGRKHIIWMNHYLVSFVIVTTATLAYFIFYHIHRLCLARIQNRRWQRLTTDLQNTFGQLQLRVVKEGDEEINPNGDSCVICFERYKPNDIVRILTCKHFFHKNCIDPWILPHGTCPICKCDILKVLGIQVVVENGTEPLQVLMSNELPETLSPSEEETNNEVSPAGTSDKVIHVEENPTSQNNDIQPHSVVEDVHPSP",
# RN135_HUMAN
"MAGLGLGSAVPVWLAEDDLGCIICQGLLDWPATLPCGHSFCRHCLEALWGARDARRWACPTCRQGAAQQPHLRKNTLLQDLADKYRRAAREIQAGSDPAHCPCPGSSSLSSAAARPRRRPELQRVAVEKSITEVAQELTELVEHLVDIVRSLQNQRPLSESGPDNELSILGKAFSSGVDLSMASPKLVTSDTAAGKIRDILHDLEEIQEKLQESVTWKEAPEAQMQGELLEAPSSSSCPLPDQSHPALRRASRFAQWAIHPTFNLKSLSCSLEVSKDSRTVTVSHRPQPYRWSCERFSTSQVLCSQALSSGKHYWEVDTRNCSHWAVGVASWEMSRDQVLGRTMDSCCVEWKGTSQLSAWHMVKETVLGSDRPGVVGIWLNLEEGKLAFYSVDNQEKLLYECTISASSPLYPAFWLYGLHPGNYLIIKQVKV",
# RN138_HUMAN
"MAEDLSAATSYTEDDFYCPVCQEVLKTPVRTTACQHVFCRKCFLTAMRESGAHCPLCRGNVTRRERACPERALDLENIMRKFSGSCRCCAKQIKFYRMRHHYKSCKKYQDEYGVSSIIPNFQISQDSVGNSNRSETSTSDNTETYQENTSSSGHPTFKCPLCQESNFTRQRLLDHCNSNHLFQIVPVTCPICVSLPWGDPSQITRNFVSHLNQRHQFDYGEFVNLQLDEETQYQTAVEESFQVNI",
# RN139_HUMAN
"MAAVGPPQQQVRMAHQQVWAALEVALRVPCLYIIDAIFNSYPDSSQSRFCIVLQIFLRLFGVFASSIVLILSQRSLFKFYTYSSAFLLAATSVLVNYYASLHIDFYGAYNTSAFGIELLPRKGPSLWMALIVLQLTFGIGYVTLLQIHSIYSQLIILDLLVPVIGLITELPLHIRETLLFTSSLILTLNTVFVLAVKLKWFYYSTRYVYLLVRHMYRIYGLQLLMEDTWKRIRFPDILRVFWLTRVTAQATVLMYILRMANETDSFFISWDDFWDLICNLIISGCDSTLTVLGMSAVISSVAHYLGLGILAFIGSTEEDDRRLGFVAPVLFFILALQTGLSGLRPEERLIRLSRNMCLLLTAVLHFIHGMTDPVLMSLSASHVSSFRRHFPVLFVSACLFILPVLLSYVLWHHYALNTWLFAVTAFCVELCLKVIVSLTVYTLFMIDGYYNVLWEKLDDYVYYVRSTGSIIEFIFGVVMFGNGAYTMMFESGSKIRAFMMCLHAYFNIYLQAKNGWKTFMNRRTAVKKINSLPEIKGSRLQEINDVCAICYHEFTTSARITPCNHYFHALCLRKWLYIQDTCPMCHQKVYIEDDIKDNSNVSNNNGFIPPNETPEEAVREAAAESDRELNEDDSTDCDDDVQRERNGVIQHTGAAAEEFNDDTD",
# RNF14_HUMAN
"MSSEDREAQEDELLALASIYDGDEFRKAESVQGGETRIYLDLPQNFKIFVSGNSNECLQNSGFEYTICFLPPLVLNFELPPDYPSSSPPSFTLSGKWLSPTQLSALCKHLDNLWEEHRGSVVLFAWMQFLKEETLAYLNIVSPFELKIGSQKKVQRRTAQASPNTELDFGGAAGSDVDQEEIVDERAVQDVESLSNLIQEILDFDQAQQIKCFNSKLFLCSICFCEKLGSECMYFLECRHVYCKACLKDYFEIQIRDGQVQCLNCPEPKCPSVATPGQVKELVEAELFARYDRLLLQSSLDLMADVVYCPRPCCQLPVMQEPGCTMGICSSCNFAFCTLCRLTYHGVSPCKVTAEKLMDLRNEYLQADEANKRLLDQRYGKRVIQKALEEMESKEWLEKNSKSCPCCGTPIEKLDGCNKMTCTGCMQYFCWICMGSLSRANPYKHFNDPGSPCFNRLFYAVDVDDDIWEDEVED",
# RN141_HUMAN
"MGQQISDQTQLVINKLPEKVAKHVTLVRESGSLTYEEFLGRVAELNDVTAKVASGQEKHLLFEVQPGSDSSAFWKVVVRVVCTKINKSSGIVEASRIMNLYQFIQLYKDITSQAAGVLAQSSTSEEPDENSSSVTSCQASLWMGRVKQLTDEEECCICMDGRADLILPCAHSFCQKCIDKWSDRHRNCPICRLQMTGANESWVVSDAPTEDDMANYILNMADEAGQPHRP",
# R144A_HUMAN
"MTTTRYRPTWDLALDPLVSCKLCLGEYPVEQMTTIAQCQCIFCTLCLKQYVELLIKEGLETAISCPDAACPKQGHLQENEIECMVAAEIMQRYKKLQFEREVLFDPCRTWCPASTCQAVCQLQDVGLQTPQPVQCKACRMEFCSTCKASWHPGQGCPETMPITFLPGETSAAFKMEEDDAPIKRCPKCKVYIERDEGCAQMMCKNCKHAFCWYCLESLDDDFLLIHYDKGPCRNKLGHSRASVIWHRTQVVGIFAGFGLLLLVASPFLLLATPFVLCCKCKCSKGDDDPLPT",
# R144B_HUMAN
"MGSAGRLHYLAMTAENPTPGDLAPAPLITCKLCLCEQSLDKMTTLQECQCIFCTACLKQYMQLAIREGCGSPITCPDMVCLNHGTLQEAEIACLVPVDQFQLYQRLKFEREVHLDPYRTWCPVADCQTVCPVASSDPGQPVLVECPSCHLKFCSCCKDAWHAEVSCRDSQPIVLPTEHRALFGTDAEAPIKQCPVCRVYIERNEGCAQMMCKNCKHTFCWYCLQNLDNDIFLRHYDKGPCRNKLGHSRASVMWNRTQVVGILVGLGIIALVTSPLLLLASPCIICCVCKSCRGKKKKHDPSTT",
# RN145_HUMAN
"MAAKEKLEAVLNVALRVPSIMLLDVLYRWDVSSFFQQIQRSSLSNNPLFQYKYLALNMHYVGYILSVVLLTLPRQHLVQLYLYFLTALLLYAGHQISRDYVRSELEFAYEGPMYLEPLSMNRFTTALIGQLVVCTLCSCVMKTKQIWLFSAHMLPLLARLCLVPLETIVIINKFAMIFTGLEVLYFLGSNLLVPYNLAKSAYRELVQVVEVYGLLALGMSLWNQLVVPVLFMVFWLVLFALQIYSYFSTRDQPASRERLLFLFLTSIAECCSTPYSLLGLVFTVSFVALGVLTLCKFYLQGYRAFMNDPAMNRGMTEGVTLLILAVQTGLIELQVVHRAFLLSIILFIVVASILQSMLEIADPIVLALGASRDKSLWKHFRAVSLCLFLLVFPAYMAYMICQFFHMDFWLLIIISSSILTSLQVLGTLFIYVLFMVEEFRKEPVENMDDVIYYVNGTYRLLEFLVALCVVAYGVSETIFGEWTVMGSMIIFIHSYYNVWLRAQLGWKSFLLRRDAVNKIKSLPIATKEQLEKHNDICAICYQDMKSAVITPCSHFFHAGCLKKWLYVQETCPLCHCHLKNSSQLPGLGTEPVLQPHAGAEQNVMFQEGTEPPGQEHTPGTRIQEGSRDNNEYIARRPDNQEGAFDPKEYPHSAKDEAHPVESA",
# RN146_HUMAN
"MMAGCGEIDHSINMLPTNRKANESCSNTAPSLTVPECAICLQTCVHPVSLPCKHVFCYLCVKGASWLGKRCALCRQEIPEDFLDKPTLLSPEELKAASRGNGEYAWYYEGRNGWWQYDERTSRELEDAFSKGKKNTEMLIAGFLYVADLENMVQYRRNEHGRRRKIKRDIIDIPKKGVAGLRLDCDANTVNLARESSADGADSVSAQSGASVQPLVSSVRPLTSVDGQLTSPATPSPDASTSLEDSFAHLQLSGDNTAERSHRGEGEEDHESPSSGRVPAPDTSIEETESDASSDSEDVSAVVAQHSLTQQRLLVSNANQTVPDRSDRSGTDRSVAGGGTVSVSVRSRRPDGQCTVTEV",
# RN148_HUMAN
"MSFLRITPSTHSSVSSGLLRLSIFLLLSFPDSNGKAIWTAHLNITFQVGNEITSELGESGVFGNHSPLERVSGVVALPEGWNQNACHPLTNFSRPKQADSWLALIERGGCTFTHKINVAAEKGANGVIIYNYQGTGSKVFPMSHQGTENIVAVMISNLKGMEILHSIQKGVYVTVIIEVGRMHMQWVSHYIMYLFTFLAATIAYFYLDCVWRLTPRVPNSFTRRRSQIKTDVKKAIDQLQLRVLKEGDEELDLNEDNCVVCFDTYKPQDVVRILTCKHFFHKACIDPWLLAHRTCPMCKCDILKT",
# RN149_HUMAN
"MAWRRREASVGARGVLALALLALALCVPGARGRALEWFSAVVNIEYVDPQTNLTVWSVSESGRFGDSSPKEGAHGLVGVPWAPGGDLEGCAPDTRFFVPEPGGRGAAPWVALVARGGCTFKDKVLVAARRNASAVVLYNEERYGNITLPMSHAGTGNIVVIMISYPKGREILELVQKGIPVTMTIGVGTRHVQEFISGQSVVFVAIAFITMMIISLAWLIFYYIQRFLYTGSQIGSQSHRKETKKVIGQLLLHTVKHGEKGIDVDAENCAVCIENFKVKDIIRILPCKHIFHRICIDPWLLDHRTCPMCKLDVIKALGYWGEPGDVQEMPAPESPPGRDPAANLSLALPDDDGSDDSSPPSASPAESEPQCDPSFKGDAGENTALLEAGRSDSRHGGPIS",
# RN150_HUMAN
"MAMSLIQACCSLALSTWLLSFCFVHLLCLDFTVAEKEEWYTAFVNITYAEPAPDPGAGAAGGGGAELHTEKTECGRYGEHSPKQDARGEVVMASSAHDRLACDPNTKFAAPTRGKNWIALIPKGNCTYRDKIRNAFLQNASAVVIFNVGSNTNETITMPHAGVEDIVAIMIPEPKGKEIVSLLERNITVTMYITIGTRNLQKYVSRTSVVFVSISFIVLMIISLAWLVFYYIQRFRYANARDRNQRRLGDAAKKAISKLQIRTIKKGDKETESDFDNCAVCIEGYKPNDVVRILPCRHLFHKSCVDPWLLDHRTCPMCKMNILKALGIPPNADCMDDLPTDFEGSLGGPPTNQITGASDTTVNESSVTLDPAVRTVGALQVVQDTDPIPQEGDVIFTTNSEQEPAVSSDSDISLIMAMEVGLSDVELSTDQDCEEVKS",
# RN151_HUMAN
"MGGGYDLNLFASPPDSNFVCSVCHGVLKRPARLPCSHIFCKKCILRWLARQKTCPCCRKEVKRKKVVHMNKLRKTIGRLEVKCKNADAGCIVTCPLAHRKGHQDSCPFELTACPNEGCTSQVPRGTLAEHRQHCQQGSQQRCPLGCGATLDPAERARHNCYRELHNAWSVRQERRRPLLLSLLRRVRWLDQATSVVRRELAELSNFLEEDTALLEGAPQEEAEAAPEGNVGAEVVGEPRANIPCK",
# RN152_HUMAN
"METLSQDSLLECQICFNYYSPRRRPKLLDCKHTCCSVCLQQMRTSQKDVRCPWCRGVTKLPPGFSVSQLPDDPEVLAVIAIPHTSEHTPVFIKLPSNGCYMLPLPISKERALLPGDMGCRLLPGSQQKSVTVVTIPAEQQPLQGGAPQEAVEEEQDRRGVVKSSTWSGVCTVILVACVLVFLLGIVLHNMSCISKRFTVISCG",
# RN157_HUMAN
"MGALTSRQHAGVEEVDIPSNSVYRYPPKSGSYFASHFIMGGEKFDSTHPEGYLFGENSDLNFLGNRPVVFPYAAPPPQEPVKTLRSLVNIRKDTLRLVKCAEEVKSPGEEASKAKVHYNVEFTFDTDARVAITIYYQATEEFQNGIASYIPKDNSLQSETVQYKRGVCQQFCLPSHTVDPSEWAEEELGFDLDREVYPLVVHAVVDEGDEYFGHCHVLLGTFEKHTDGTFCVKPLKQKQVVDGVSYLLQEIYGIENKYNTQDSKVAEDEVSDNSAECVVCLSDVRDTLILPCRHLCLCNTCADTLRYQANNCPICRLPFRALLQIRAMRKKLGPLSPTSFNPIISSQTSDSEEHPSSENIPPGYEVVSLLEALNGPLTPSPAVPPLHVLGDGHLSGMLPSYGSDGHLPPVRTISPLDRLSDSSSQGLKLKKSLSKSTSQNSSVLHEEEDEHSCSESETQLSQRPSVQHLGEECGVTPESENLTLSSSGAIDQSSCTGTPLSSTISSPEGPASSSLAQSVMSMASSQISTDTVSSMSGSYIAPGTEEEGEALSSPQPASRAPSEEGEGLPAESPDSNFAGLPAGEQDAEGNDVIEEEDGSPTQEGQRTCAFLGMECDNNNDFDIASVKALDNKLCSEVCLPGAWQADDNAVSRNAQRRRLSSSSLEDSETRPCVWGPLAV",
# ARK2C_HUMAN
"MVLVHVGYLVLPVFGSVRNRGAPFQRSQHPHATSCRHFHLGPPQPQQLAPDFPLAHPVQSQPGLSAHMAPAHQHSGALHQSLTPLPTLQFQDVTGPSFLPQALHQQYLLQQQLLEAQHRRLVSHPRRSQERVSVHPHRLHPSFDFGQLQTPQPRYLAEGTDWDLSVDAGLSPAQFQVRPIPQHYQHYLATPRMHHFPRNSSSTQMVVHEIRNYPYPQLHFLALQGLNPSRHTSAVRESYEELLQLEDRLGNVTRGAVQNTIERFTFPHKYKKRRPQDGKGKKDEGEESDTDEKCTICLSMLEDGEDVRRLPCMHLFHQLCVDQWLAMSKKCPICRVDIETQLGADS",
# RN166_HUMAN
"MAMFRSLVASAQQRQPPAGPAGGDSGLEAQYTCPICLEVYHRPVAIGSCGHTFCGECLQPCLQVPSPLCPLCRLPFDPKKVDKATHVEKQLSSYKAPCRGCNKKVTLAKMRVHISSCLKVQEQMANCPKFVPVVPTSQPIPSNIPNRSTFACPYCGARNLDQQELVKHCVESHRSDPNRVVCPICSAMPWGDPSYKSANFLQHLLHRHKFSYDTFVDYSIDEEAAFQAALALSLSEN",
# RN167_HUMAN
"MHPAAFPLPVVVAAVLWGAAPTRGLIRATSDHNASMDFADLPALFGATLSQEGLQGFLVEAHPDNACSPIAPPPPAPVNGSVFIALLRRFDCNFDLKVLNAQKAGYGAAVVHNVNSNELLNMVWNSEEIQQQIWIPSVFIGERSSEYLRALFVYEKGARVLLVPDNTFPLGYYLIPFTGIVGLLVLAMGAVMIARCIQHRKRLQRNRLTKEQLKQIPTHDYQKGDQYDVCAICLDEYEDGDKLRVLPCAHAYHSRCVDPWLTQTRKTCPICKQPVHRGPGDEDQEEETQGQEEGDEGEPRDHPASERTPLLGSSPTLPTSFGSLAPAPLVFPGPSTDPPLSPPSSPVILV",
# RN168_HUMAN
"MALPKDAIPSLSECQCGICMEILVEPVTLPCNHTLCKPCFQSTVEKASLCCPFCRRRVSSWTRYHTRRNSLVNVELWTIIQKHYPRECKLRASGQESEEVADDYQPVRLLSKPGELRREYEEEISKVAAERRASEEEENKASEEYIQRLLAEEEEEEKRQAEKRRRAMEEQLKSDEELARKLSIDINNFCEGSISASPLNSRKSDPVTPKSEKKSKNKQRNTGDIQKYLTPKSQFGSASHSEAVQEVRKDSVSKDIDSSDRKSPTGQDTEIEDMPTLSPQISLGVGEQGADSSIESPMPWLCACGAEWYHEGNVKTRPSNHGKELCVLSHERPKTRVPYSKETAVMPCGRTESGCAPTSGVTQTNGNNTGETENEESCLLISKEISKRKNQESSFEAVKDPCFSAKRRKVSPESSPDQEETEINFTQKLIDLEHLLFERHKQEEQDRLLALQLQKEVDKEQMVPNRQKGSPDEYHLRATSSPPDKVLNGQRKNPKDGNFKRQTHTKHPTPERGSRDKNRQVSLKMQLKQSVNRRKMPNSTRDHCKVSKSAHSLQPSISQKSVFQMFQRCTK",
# RN169_HUMAN
"MAAAGPSTRASSAAAAAALSRRGRRGRCDETAAAKTGAPGPASGPSLLVLSPPLLQPPLPPRPEESGCAGCLEPPGEAAALPCGHSLCRGCAQRAADAAGPGCPRCRARGPGWARRRARDDGQADSEVLGECARRSQPERCRPRRDGGAAAAGPRPEQEPRAAPAEPDFIFRAPIKLSKPGELREEYESLRKLREEKLQEEKPSEDQIHKLLPEDTETGKRKMDEQKKRDEPLVLKTNLERCPARLSDSENEEPSRGQMTQTHRSAFVSKNNSYSLAFLAGKLNSKVERSQSCSDTAQERAKSRVRAVPGNKAKVTTMTPASNPIIGVLLSTQNNRCVSAPDLTIEKRLPFSSLSSLASLHKPERSVSPESNDSISEELNHFKPIVCSPCTPPKRLPDGRVLSPLIIKSTPRNLNRSLQKQTSYEASPRILKKWEQIFQERQIKKTLSKATLTSLAPEMGEELLGSEGIHSSKEKPLVAVNTRLSGGQVLSEYTGPTSADLDHFPSVSQTKAEQDSDNKSSTEIPLETCCSSELKGGGSGTSLEREQFEGLGSTPDAKLDKTCISRAMKITTVNSVLPQNSVLGGVLKTKQQLKTLNHFDLTNGVLVESLSEEPLPSLRRGRKRHCKTKHLEQNGSLKKLRQTSGEVGLAPTDPVLREMEQKLQQEEEDRQLALQLQRMFDNERRTVSRRKGSVDQYLLRSSNMAGAK",
# RNF17_HUMAN
"MAAEASKTGPSRSSYQRMGRKSQPWGAAEIQCTRCGRRVSRSSGHHCELQCGHAFCELCLLMTEECTTIICPDCEVATAVNTRQRYYPMAGYIKEDSIMEKLQPKTIKNCSQDFKKTADQLTTGLERSASTDKTLLNSSAVMLDTNTAEEIDEALNTAHHSFEQLSIAGKALEHMQKQTIEERERVIEVVEKQFDQLLAFFDSRKKNLCEEFARTTDDYLSNLIKAKSYIEEKKNNLNAAMNIARALQLSPSLRTYCDLNQIIRTLQLTSDSELAQVSSPQLRNPPRLSVNCSEIICMFNNMGKIEFRDSTKCYPQENEIRQNVQKKYNNKKELSCYDTYPPLEKKKVDMSVLTSEAPPPPLQPETNDVHLEAKNFQPQKDVATASPKTIAVLPQMGSSPDVIIEEIIEDNVESSAELVFVSHVIDPCHFYIRKYSQIKDAKVLEKKVNEFCNRSSHLDPSDILELGARIFVSSIKNGMWCRGTITELIPIEGRNTRKPCSPTRLFVHEVALIQIFMVDFGNSEVLIVTGVVDTHVRPEHSAKQHIALNDLCLVLRKSEPYTEGLLKDIQPLAQPCSLKDIVPQNSNEGWEEEAKVEFLKMVNNKAVSMKVFREEDGVLIVDLQKPPPNKISSDMPVSLRDALVFMELAKFKSQSLRSHFEKNTTLHYHPPILPKEMTDVSVTVCHINSPGDFYLQLIEGLDILFLLKTIEEFYKSEDGENLEILCPVQDQACVAKFEDGIWYRAKVIGLPGHQEVEVKYVDFGNTAKITIKDVRKIKDEFLNAPEKAIKCKLAYIEPYKRTMQWSKEAKEKFEEKAQDKFMTCSVIKILEDNVLLVELFDSLGAPEMTTTSINDQLVKEGLASYEIGYILKDNSQKHIEVWDPSPEEIISNEVHNLNPVSAKSLPNENFQSLYNKELPVHICNVISPEKIYVQWLLTENLLNSLEEKMIAAYENSKWEPVKWENDMHCAVKIQDKNQWRRGQIIRMVTDTLVEVLLYDVGVELVVNVDCLRKLEENLKTMGRLSLECSLVDIRPAGGSDKWTATACDCLSLYLTGAVATIILQVDSEENNTTWPLPVKIFCRDEKGERVDVSKYLIKKGLALRERRINNLDNSHSLSEKSLEVPLEQEDSVVTNCIKTNFDPDKKTADIISEQKVSEFQEKILEPRTTRGYKPPAIPNMNVFEATVSCVGDDGTIFVVPKLSEFELIKMTNEIQSNLKCLGLLEPYFWKKGEACAVRGSDTLWYRGKVMEVVGGAVRVQYLDHGFTEKIPQCHLYPILLYPDIPQFCIPCQLHNTTPVGNVWQPDAIEVLQQLLSKRQVDIHIMELPKNPWEKLSIHLYFDGMSLSYFMAYYKYCTSEHTEEMLKEKPRSDHDKKYEEEQWEIRFEELLSAETDTPLLPPYLSSSLPSPGELYAVQVKHVVSPNEVYICLDSIETSNQSNQHSDTDDSGVSGESESESLDEALQRVNKKVEALPPLTDFRTEMPCLAEYDDGLWYRAKIVAIKEFNPLSILVQFVDYGSTAKLTLNRLCQIPSHLMRYPARAIKVLLAGFKPPLRDLGETRIPYCPKWSMEALWAMIDCLQGKQLYAVSMAPAPEQIVTLYDDEQHPVHMPLVEMGLADKDE",
# RN170_HUMAN
"MAKYQGEVQSLKLDDDSVIEGVSDQVLVAVVVSFALIATLVYALFRNVHQNIHPENQELVRVLREQLQTEQDAPAATRQQFYTDMYCPICLHQASFPVETNCGHLFCGACIIAYWRYGSWLGAISCPICRQTVTLLLTVFGEDDQSQDVLRLHQDINDYNRRFSGQPRSIMERIMDLPTLLRHAFREMFSVGGLFWMFRIRIILCLMGAFFYLISPLDFVPEALFGILGFLDDFFVIFLLLIYISIMYREVITQRLTR",
# RN175_HUMAN
"MAAGTAARKAAPVLEAPPQQEQLSHTKLSAEDTWNLQQERMYKMHRGHDSMHVEMILIFLCVLVIAQIVLVQWRQRHGRSYNLVTLLQMWVVPLYFTIKLYWWRFLSMWGMFSVITSYILFRATRKPLSGRTPRLVYKWFLLIYKLSYAFGVVGYLAIMFTMCGFNLFFKIKARDSMDFGIVSLFYGLYYGVMGRDFAEICSDYMASTIGFYSVSRLPTRSLSDNICAVCGQKIIVELDEEGLIENTYQLSCNHVFHEFCIRGWCIVGKKQTCPYCKEKVDLKRMISNPWERTHFLYGQILDWLRYLVAWQPVVIGIVQGIIYSLGLE",
# RN180_HUMAN
"MKRSKELITKNHSQEETSILRCWKCRKCIASSGCFMEYLENQVIKDKDDSVDAQNICHVWHMNVEALPEWISCLIQKAQWTVGKLNCPFCGARLGGFNFVSTPKCSCGQLAAVHLSKSRTDYQPTQAGRLMRPSVKYLSHPRVQSGCDKEALLTGGGSENRNHRLLNMARNNNDPGRLTEALCLEVRPTYFEMKNEKLLSKASEPKYQLFVPQLVTGRCATRAFHRKSHSLDLNISEKLTLLPTLYEIHSKTTAYSRLNETQPIDLSGLPLQSSKNSYSFQNPSSFDPSMLLQRFSVAPHETQTQRGGEFQCGLEAASVYSDHTNTNNLTFLMDLPSAGRSMPEASDQEEHLSPLDFLHSANFSLGSINQRLNKRERSKLKNLRRKQRRRERWLQKQGKYSGVGLLDHMTLNNEMSTDEDNEYAEEKDSYICAVCLDVYFNPYMCYPCHHIFCEPCLRTLAKDNPSSTPCPLCRTIISRVFFQTELNNATKTFFTKEYLKIKQSFQKSNSAKWPLPSCRKAFHLFGGFRRHAAPVTRRQFPHGAHRMDYLHFEDDSRGWWFDMDMVIIYIYSVNWVIGFIVFCFLCYFFFPF",
# RN181_HUMAN
"MASYFDEHDCEPSDPEQETRTNMLLELARSLFNRMDFEDLGLVVDWDHHLPPPAAKTVVENLPRTVIRGSQAELKCPVCLLEFEEEETAIEMPCHHLFHSSCILPWLSKTNSCPLCRYELPTDDDTYEEHRRDKARKQQQQHRLENLHGAMYT",
# RN182_HUMAN
"MASQPPEDTAESQASDELECKICYNRYNLKQRKPKVLECCHRVCAKCLYKIIDFGDSPQGVIVCPFCRFETCLPDDEVSSLPDDNNILVNLTCGGKGKKCLPENPTELLLTPKRLASLVSPSHTSSNCLVITIMEVQRESSPSLSSTPVVEFYRPASFDSVTTVSHNWTVWNCTSLLFQTSIRVLVWLLGLLYFSSLPLGIYLLVSKKVTLGVVFVSLVPSSLVILMVYGFCQCVCHEFLDCMAPPS",
# RN183_HUMAN
"MAEQQGRELEAECPVCWNPFNNTFHTPKMLDCCHSFCVECLAHLSLVTPARRRLLCPLCRQPTVLASGQPVTDLPTDTAMLALLRLEPHHVILEGHQLCLKDQPKSRYFLRQPQVYTLDLGPQPGGQTGPPPDTASATVSTPILIPSHHSLRECFRNPQFRIFAYLMAVILSVTLLLIFSIFWTKQFLWGVG",
# RN185_HUMAN
"MASKGPSASASPENSSAGGPSGSSNGAGESGGQDSTFECNICLDTAKDAVISLCGHLFCWPCLHQWLETRPNRQVCPVCKAGISRDKVIPLYGRGSTGQQDPREKTPPRPQGQRPEPENRGGFQGFGFGDGGFQMSFGIGAFPFGIFATAFNINDGRPPPAVPGTPQYVDEQFLSRLFLFVALVIMFWLLIA",
# RN186_HUMAN
"MACTKTLQQSQPISAGATTTTTAVAPAGGHSGSTECDLECLVCREPYSCPRLPKLLACQHAFCAICLKLLLCVQDNTWSITCPLCRKVTAVPGGLICSLRDHEAVVGQLAQPCTEVSLCPQGLVDPADLAAGHPSLVGEDGQDEVSANHVAARRLAAHLLLLALLIILIGPFIYPGVLRWVLTFIIALALLMSTLFCCLPSTRGSCWPSSRTLFCREQKHSHISSIA",
# RN187_HUMAN
"MALPAGPAEAACALCQRAPREPVRADCGHRFCRACVVRFWAEEDGPFPCPECADDCWQRAVEPGRPPLSRRLLALEEAAAAPARDGPASEAALQLLCRADAGPLCAACRMAAGPEPPEWEPRWRKALRGKENKGSVEIMRKDLNDARDLHGQAESAAAVWKGHVMDRRKKALTDYKKLRAFFVEEEEHFLQEAEKEEGLPEDELADPTERFRSLLQAVSELEKKHRNLGLSMLLQ",
# RN19A_HUMAN
"MQEQEIGFISKYNEGLCVNTDPVSILTSILDMSLHRQMGSDRDLQSSASSVSLPSVKKAPKKRRISIGSLFRRKKDNKRKSRELNGGVDGIASIESIHSEMCTDKNSIFSTNTSSDNGLTSISKQIGDFIECPLCLLRHSKDRFPDIMTCHHRSCVDCLRQYLRIEISESRVNISCPECTERFNPHDIRLILSDDVLMEKYEEFMLRRWLVADPDCRWCPAPDCGYAVIAFGCASCPKLTCGREGCGTEFCYHCKQIWHPNQTCDAARQERAQSLRLRTIRSSSISYSQESGAAADDIKPCPRCAAYIIKMNDGSCNHMTCAVCGCEFCWLCMKEISDLHYLSPSGCTFWGKKPWSRKKKILWQLGTLVGAPVGIALIAGIAIPAMIIGIPVYVGRKIHNRYEGKDVSKHKRNLAIAGGVTLSVIVSPVVAAVTVGIGVPIMLAYVYGVVPISLCRSGGCGVSAGNGKGVRIEFDDENDINVGGTNTAVDTTSVAEARHNPSIGEGSVGGLTGSLSASGSHMDRIGAIRDNLSETASTMALAGASITGSLSGSAMVNCFNRLEVQADVQKERYSLSGESGTVSLGTVSDNASTKAMAGSILNSYIPLDKEGNSMEVQVDIESKPSKFRHNSGSSSVDDGSATRSHAGGSSSGLPEGKSSATKWSKEATAGKKSKSGKLRKKGNMKINETREDMDAQLLEQQSTNSSEFEAPSLSDSMPSVADSHSSHFSEFSCSDLESMKTSCSHGSSDYHTRFATVNILPEVENDRLENSPHQCSISVVTQTASCSEVSQLNHIAEEHGNNGIKPNVDLYFGDALKETNNNHSHQTMELKVAIQTEI",
# RN19B_HUMAN
"MGSEKDSESPRSTSLHAAAPDPKCRSGGRRRRLTLHSVFSASARGRRARAKPQAEPPPPAAQPPPAPAPAAAQGPPPEALPAEPAAEAEAEAAAAAAEPGFDDEEAAEGGGPGAEEVECPLCLVRLPPERAPRLLSCPHRSCRDCLRHYLRLEISESRVPISCPECSERLNPHDIRLLLADPPLMHKYEEFMLRRYLASDPDCRWCPAPDCGYAVIAYGCASCPKLTCEREGCQTEFCYHCKQIWHPNQTCDMARQQRAQTLRVRTKHTSGLSYGQESGPADDIKPCPRCSAYIIKMNDGSCNHMTCAVCGCEFCWLCMKEISDLHYLSPSGCTFWGKKPWSRKKKILWQLGTLIGAPVGISLIAGIAIPAMVIGIPVYVGRKIHSRYEGRKTSKHKRNLAITGGVTLSVIASPVIAAVSVGIGVPIMLAYVYGVVPISLCRGGGCGVSTANGKGVKIEFDEDDGPITVADAWRALKNPSIGESSIEGLTSVLSTSGSPTDGLSVMQGPYSETASFAALSGGTLSGGILSSGKGKYSRLEVQADVQKEIFPKDTASLGAISDNASTRAMAGSIISSYNPQDRECNNMEIQVDIEAKPSHYQLVSGSSTEDSLHVHAQMAENEEEGSGGGGSEEDPPCRHQSCEQKDCLASKPWDISLAQPESIRSDLESSDAQSDDVPDITSDECGSPRSHTAACPSTPRAQGAPSPSAHMNLSALAEGQTVLKPEGGEARV",
# RING2_HUMAN
"MSQAVQTNGTQPLSKTWELSLYELQRTPQEAITDGLEIVVSPRSLHSELMCPICLDMLKNTMTTKECLHRFCADCIITALRSGNKECPTCRKKLVSKRSLRPDPNFDALISKIYPSRDEYEAHQERVLARINKHNNQQALSHSIEEGLKIQAMNRLQRGKKQQIENGSGAEDNGDSSHCSNASTHSNQEAGPSNKRTKTSDDSGLELDNNNAAMAIDPVMDGASEIELVFRPHPTLMEKDDSAQTRYIKTSGNATVDHLSKYLAVRLALEELRSKGESNQMNLDTASEKQYTIYIATASGQFTVLNGSFSLELVSEKYWKVNKPMELYYAPTKEHK",
# BRE1A_HUMAN
"MSGIGNKRAAGEPGTSMPPEKKAAVEDSGTTVETIKLGGVSSTEELDIRTLQTKNRKLAEMLDQRQAIEDELREHIEKLERRQATDDASLLIVNRYWSQFDENIRIILKRYDLEQGLGDLLTERKALVVPEPEPDSDSNQERKDDRERGEGQEPAFSFLATLASSSSEEMESQLQERVESSRRAVSQIVTVYDKLQEKVELLSRKLNSGDNLIVEEAVQELNSFLAQENMRLQELTDLLQEKHRTMSQEFSKLQSKVETAESRVSVLESMIDDLQWDIDKIRKREQRLNRHLAEVLERVNSKGYKVYGAGSSLYGGTITINARKFEEMNAELEENKELAQNRLCELEKLRQDFEEVTTQNEKLKVELRSAVEQVVKETPEYRCMQSQFSVLYNESLQLKAHLDEARTLLHGTRGTHQHQVELIERDEVSLHKKLRTEVIQLEDTLAQVRKEYEMLRIEFEQTLAANEQAGPINREMRHLISSLQNHNHQLKGEVLRYKRKLREAQSDLNKTRLRSGSALLQSQSSTEDPKDEPAELKPDSEDLSSQSSASKASQEDANEIKSKRDEEERERERREKEREREREREKEKEREREKQKLKESEKERDSAKDKEKGKHDDGRKKEAEIIKQLKIELKKAQESQKEMKLLLDMYRSAPKEQRDKVQLMAAEKKSKAELEDLRQRLKDLEDKEKKENKKMADEDALRKIRAVEEQIEYLQKKLAMAKQEEEALLSEMDVTGQAFEDMQEQNIRLMQQLREKDDANFKLMSERIKSNQIHKLLKEEKEELADQVLTLKTQVDAQLQVVRKLEEKEHLLQSNIGTGEKELGLRTQALEMNKRKAMEAAQLADDLKAQLELAQKKLHDFQDEIVENSVTKEKDMFNFKRAQEDISRLRRKLETTKKPDNVPKCDEILMEEIKDYKARLTCPCCNMRKKDAVLTKCFHVFCFECVKTRYDTRQRKCPKCNAAFGANDFHRIYIG",
# RN207_HUMAN
"MSGAIFGPLEGPSSLDAPSIHPLVCPLCHVQYERPCLLDCFHDFCAGCLRGRATDGRLTCPLCQHQTVLKGPSGLPPVDRLLQFLVDSSGDGVEAVRCANCDLECSEQDVETTYFCNTCGQPLCARCRDETHRARMFARHDIVALGQRSRDVPQKCTLHAEPYLLFSTDKKLLLCIRCFRDMQKESRAHCVDLESAYVQGCERLEQAVLAVKALQTATREAIALLQAMVEEVRHSAAEEEDAIHALFGSMQDRLAERKALLLQAVQSQYEEKDKAFKEQLSHLATLLPTLQVHLVICSSFLSLANKAEFLDLGYELMERLQGIVTRPHHLRPIQSSKIASDHRAEFARCLEPLLLLGPRRVAAAASGANTLAGGLGPKALTGPHCPSPVGKMSGSPVQKPTLHRSISTKVLLAEGENTPFAEHCRHYEDSYRHLQAEMQSLKDQVQELHRDLTKHHSLIKAEIMGDVLHKSLQLDVQIASEHASLEGMRVVFQEIWEEAYQRVANEQEIYEAQLHDLLQLRQENAYLTTITKQITPYVRSIAKVKERLEPRFQAPVDEQSESLQNTHDDSRNNAASARNNPGSVPEKREKTSEPKGNSWAPNGLSEEPLLKNMDHHRSKQKNGGDVPTWREHPT",
# RN208_HUMAN
"MPSDPGPEAGSGWPGLLMSCLKGPHVILKMEAMKIVHPEKFPELPAAPCFPPAPRPTPTLAPKRAWPSDTEIIVNQACGGDMPALEGAPHTPPLPRRPRKGSSELGFPRVAPEDEVIVNQYVIRPGPSASAASSAAAGEPLECPTCGHSYNVTQRRPRVLSCLHSVCEQCLQILYESCPKYKFISCPTCRRETVLFTDYGLAALAVNTSILSRLPPEALTAPSGGQWGAEPEGSCYQTFRQYCGAACTCHVRNPLSACSIM",
# RN212_HUMAN
"MANWVFCNRCFQPPHRTSCFSLTNCGHVYCDACLGKGKKNECLICKAPCRTVLLSKHTDADIQAFFMSIDSLCKKYSRETSQILEFQEKHRKRLLAFYREKISRLEESLRKSVLQIEQLQSMRSSQQTAFSTIKSSVSTKPHGCLLPPHSSAPDRLESMEVDLSPSPIRKSEIAAGPARISMISPPQDGRMGPHLTASFCFIPWLTLSKPPVPGECVISRGSPCFCIDVCPHWLLLLAFSSGRHGELTNSKTLPIYAEVQRAVLFPFQQAEGTLDTFRTPAVSVVFPLCQFERKKSF",
# R212B_HUMAN
"MDWFHCNQCFRKDGAHFFVTSCGHIFCKKCVTLEKCAVCGTACKHLALSDNLKPQEKMFFKSPVETALQYFSHISQVWSFQKKQTDLLIAFYKHRITKLETAMQEAQQALVSQDKELSVLRKENGELKKFLAILKESPSRYQGSRSITPRPVGITSPSQSVTPRPSFQHSSQVVSRSSSAESIPYREAGFGSLGQGGRGLQGRRTPRDSYNETPSPASTHSLSYRTSSASSGQGIFSFRPSPNGHSGHTRVLTPNNFAQRESTTTLESLPSFQLPVLQTLYQQRRHMGLPSGREAWTTSR",
# RN213_HUMAN
"MECPSCQHVSKEETPKFCSQCGERLPPAAPIADSENNNSTMASASEGEMECGQELKEEGGPCLFPGSDSWQENPEEPCSKASWTVQESKKKKRKKKKKGNKSASSELASLPLSPASPCHLTLLSNPWPQDTALPHSQAQQSGPTGQPSQPPGTATTPLEGDGLSAPTEVGDSPLQAQALGEAGVATGSEAQSSPQFQDHTEGEDQDASIPSGGRGLSQEGTGPPTSAGEGHSRTEDAAQELLLPESKGGSSEPGTELQTTEQQAGASASMAVDAVAEPANAVKGAGKEMKEKTQRMKQPPATTPPFKTHCQEAETKTKDEMAAAEEKVGKNEQGEPEDLKKPEGKNRSAAAVKNEKEQKNQEADVQEVKASTLSPGGGVTVFFHAIISLHFPFNPDLHKVFIRGGEEFGESKWDSNICELHYTRDLGHDRVLVEGIVCISKKHLDKYIPYKYVIYNGESFEYEFIYKHQQKKGEYVNRCLFIKSSLLGSGDWHQYYDIVYMKPHGRLQKVMNHITDGPRKDLVKGKQIAAALMLDSTFSILQTWDTINLNSFFTQFEQFCFVLQQPMIYEGQAQLWTDLQYREKEVKRYLWQHLKKHVVPLPDGKSTDFLPVDCPVRSKLKTGLIVLFVVEKIELLLEGSLDWLCHLLTSDASSPDEFHRDLSHILGIPQSWRLYLVNLCQRCMDTRTYTWLGALPVLHCCMELAPRHKDAWRQPEDTWAALEGLSFSPFREQMLDTSSLLQFMREKQHLLSIDEPLFRSWFSLLPLSHLVMYMENFIEHLGRFPAHILDCLSGIYYRLPGLEQVLNTQDVQDVQNVQNILEMLLRLLDTYRDKIPEEALSPSYLTVCLKLHEAICSSTKLLKFYELPALSAEIVCRMIRLLSLVDSAGQRDETGNNSVQTVFQGTLAATKRWLREVFTKNMLTSSGASFTYVKEIEVWRRLVEIQFPAEHGWKESLLGDMEWRLTKEEPLSQITAYCNSCWDTKGLEDSVAKTFEKCIIEAVSSACQSQTSILQGFSYSDLRKFGIVLSAVITKSWPRTADNFNDILKHLLTLADVKHVFRLCGTDEKILANVTEDAKRLIAVADSVLTKVVGDLLSGTILVGQLELIIKHKNQFLDIWQLREKSLSPQDEQCAVEEALDWRREELLLLKKEKRCVDSLLKMCGNVKHLIQVDFGVLAVRHSQDLSSKRLNDTVTVRLSTSSNSQRATHYHLSSQVQEMAGKIDLLRDSHIFQLFWREAAEPLSEPKEDQEAAELLSEPEEESERHILELEEVYDYLYQPSYRKFIKLHQDLKSGEVTLAEIDVIFKDFVNKYTDLDSELKIMCTVDHQDQRDWIKDRVEQIKEYHHLHQAVHAAKVILQVKESLGLNGDFSVLNTLLNFTDNFDDFRRETLDQINQELIQAKKLLQDISEARCKGLQALSLRKEFICWVREALGGINELKVFVDLASISAGENDIDVDRVACFHDAVQGYASLLFKLDPSVDFSAFMKHLKKLWKALDKDQYLPRKLCDSARNLEWLKTVNESHGSVERSSLTLATAINQRGIYVIQAPKGGQKISPDTVLHLILPESPGSHEESREYSLEEVKELLNKLMLMSGKKDRNNTEVERFSEVFCSVQRLSQAFIDLHSAGNMLFRTWIAMAYCSPKQGVSLQMDFGLDLVTELKEGGDVTELLAALCRQMEHFLDSWKRFVTQKRMEHFYLNFYTAEQLVYLSTELRKQPPSDAALTMLSFIKSNCTLRDVLRASVGCGSEAARYRMRRVMEELPLMLLSEFSLVDKLRIIMEQSMRCLPAFLPDCLDLETLGHCLAHLAGMGGSPVERCLPRGLQVGQPNLVVCGHSEVLPAALAVYMQTPSQPLPTYDEVLLCTPATTFEEVALLLRRCLTLGSLGHKVYSLLFADQLSYEVARQAEELFHNLCTQQHREDYQLVMVCDGDWEHCYLPSAFSQHKVFVTPQAPLEAIQAYLAGHYRVPKQTLSAAAVFNDRLCVGIVASERAGVGKSLYVKRLHDKMKMQLNVKNVPLKTIRLIDPQVDESRVLGALLPFLDAQYQKVPVLFHLDVTSSVQTGIWVFLFKLLILQYLMDINGKMWLRNPCHLYIVEILERRTSVPSRSSSALRTRVPQFSFLDIFPKVTCRPPKEVIDMELSALRSDTEPGMDLWEFCSETFQRPYQYLRRFNQNQDLDTFQYQEGSVEGTPEECLQHFLFHCGVINPSWSELRNFARFLNYQLRDCEASLFCNPSFIGDTLRGFKKFVVTFMIFMARDFATPSLHTSDQSPGKHMVTMDGVREEDLAPFSLRKRWESEPHPYVFFNDDHTTMTFIGFHLQPNINGSVDAISHLTGKVIKRDVMTRDLYQGLLLQRVPFNVDFDKLPRHKKLERLCLTLGIPQATDPDKTYELTTDNMLKILAIEMRFRCGIPVIIMGETGCGKTRLIKFLSDLRRGGTNADTIKLVKVHGGTTADMIYSRVREAENVAFANKDQHQLDTILFFDEANTTEAISCIKEVLCDHMVDGQPLAEDSGLHIIAACNPYRKHSEEMICRLESAGLGYRVSMEETADRLGSIPLRQLVYRVHALPPSLIPLVWDFGQLSDVAEKLYIQQIVQRLVESISLDENGTRVITEVLCASQGFMRKTEDECSFVSLRDVERCVKVFRWFHEHSAMLLAQLNAFLSKSSVSKNHTERDPVLWSLMLAIGVCYHASLEKKDSYRKAIARFFPKPYDDSRLLLDEITRAQDLFLDGVPLRKTIAKNLALKENVFMMVVCIELKIPLFLVGKPGSSKSLAKTIVADAMQGPAAYSDLFRSLKQVHLVSFQCSPHSTPQGIISTFRQCARFQQGKDLQQYVSVVVLDEVGLAEDSPKMPLKTLHPLLEDGCIEDDPAPHKKVGFVGISNWALDPAKMNRGIFVSRGSPNETELIESAKGICSSDILVQDRVQGYFASFAKAYETVCKRQDKEFFGLRDYYSLIKMVFAAAKASNRKPSPQDIAQAVLRNFSGKDDIQALDIFLANLPEAKCSEEVSPMQLIKQNIFGPSQKVPGGEQEDAESRYLLVLTKNYVALQILQQTFFEGDQQPEIIFGSGFPKDQEYTQLCRNINRVKICMETGKMVLLLNLQNLYESLYDALNQYYVHLGGQKYVDLGLGTHRVKCRVHPNFRLIVIEEKDVVYKHFPIPLINRLEKHYLDINTVLEKWQKSIVEELCAWVEKFINVKAHHFQKRHKYSPSDVFIGYHSDACASVVLQVIERQGPRALTEELHQKVSEEAKSILLNCATPDAVVRLSAYSLGGFAAEWLSQEYFHRQRHNSFADFLQAHLHTADLERHAIFTEITTFSRLLTSHDCEILESEVTGRAPKPTLLWLQQFDTEYSFLKEVRNCLTNTAKCKILIFQTDFEDGIRSAQLIASAKYSVINEINKIRENEDRIFVYFITKLSRVGRGTAYVGFHGGLWQSVHIDDLRRSTLMVSDVTRLQHVTISQLFAPGDLPELGLEHRAEDGHEEAMETEASTSGEVAEVAEEAMETESSEKVGKETSELGGSDVSILDTTRLLRSCVQSAVGMLRDQNESCTRNMRRVVLLLGLLNEDDACHASFLRVSKMRLSVFLKKQEESQFHPLEWLAREACNQDALQEAGTFRHTLWKRVQGAVTPLLASMISFIDRDGNLELLTRPDTPPWARDLWMFIFSDTMLLNIPLVMNNERHKGEMAYIVVQNHMNLSENASNNVPFSWKIKDYLEELWVQAQYITDAEGLPKKFVDIFQQTPLGRFLAQLHGEPQQELLQCYLKDFILLTMRVSTEEELKFLQMALWSCTRKLKAASEAPEEEVSLPWVHLAYQRFRSRLQNFSRILTIYPQVLHSLMEARWNHELAGCEMTLDAFAAMACTEMLTRNTLKPSPQAWLQLVKNLSMPLELICSDEHMQGSGSLAQAVIREVRAQWSRIFSTALFVEHVLLGTESRVPELQGLVTEHVFLLDKCLRENSDVKTHGPFEAVMRTLCECKETASKTLSRFGIQPCSICLGDAKDPVCLPCDHVHCLRCLRAWFASEQMICPYCLTALPDEFSPAVSQAHREAIEKHARFRQMCNSFFVDLVSTICFKDNAPPEKEVIESLLSLLFVQKGRLRDAAQRHCEHTKSLSPFNDVVDKTPVIRSVILKLLLKYSFHDVKDYIQEYLTLLKKKAFITEDKTELYMLFINCLEDSILEKTSAYSRNDELNHLEEEGRFLKAYSPASRGREPANEASVEYLQEVARIRLCLDRAADFLSEPEGGPEMAKEKQCYLQQVKQFCIRVENDWHRVYLVRKLSSQRGMEFVQGLSKPGRPHQWVFPKDVVKQQGLRQDHPGQMDRYLVYGDEYKALRDAVAKAVLECKPLGIKTALKACKTPQSQQSAYFLLTLFREVAILYRSHNASLHPTPEQCEAVSKFIGECKILSPPDISRFATSLVDNSVPLLRAGPSDSNLDGTVTEMAIHAAAVLLCGQNELLEPLKNLAFSPATMAHAFLPTMPEDLLAQARRWKGLERVHWYTCPNGHPCSVGECGRPMEQSICIDCHAPIGGIDHKPRDGFHLVKDKADRTQTGHVLGNPQRRDVVTCDRGLPPVVFLLIRLLTHLALLLGASQSSQALINIIKPPVRDPKGFLQQHILKDLEQLAKMLGHSADETIGVVHLVLRRLLQEQHQLSSRRLLNFDTELSTKEMRNNWEKEIAAVISPELEHLDKTLPTMNNLISQDKRISSNPVAKIIYGDPVTFLPHLPRKSVVHCSKIWSCRKRITVEYLQHIVEQKNGKERVPILWHFLQKEAELRLVKFLPEILALQRDLVKQFQNVQQVEYSSIRGFLSKHSSDGLRQLLHNRITVFLSTWNKLRRSLETNGEINLPKDYCSTDLDLDTEFEILLPRRRGLGLCATALVSYLIRLHNEIVYAVEKLSKENNSYSVDAAEVTELHVISYEVERDLTPLILSNCQYQVEEGRETVQEFDLEKIQRQIVSRFLQGKPRLSLKGIPTLVYRHDWNYEHLFMDIKNKMAQDSLPSSVISAISGQLQSYSDACEVLSVVEVTLGFLSTAGGDPNMQLNVYTQDILQMGDQTIHVLKALNRCQLKHTIALWQFLSAHKSEQLLRLHKEPFGEISSRYKADLSPENAKLLSTFLNQTGLDAFLLELHEMIILKLKNPQTQTEERFRPQWSLRDTLVSYMQTKESEILPEMASQFPEEILLASCVSVWKTAAVLKWNREMR",
# RN214_HUMAN
"MAASEVAGVVANAPSPPESSSLCASKSDEGLPDGLSTKDSAQKQKNSPLLSVSSQTITKENNRNVHLEHSEQNPGSSAGDTSAAHQVVLGENLIATALCLSGSGSQSDLKDVASTAGEEGDTSLRESLHPVTRSLKAGCHTKQLASRNCSEEKSPQTSILKEGNRDTSLDFRPVVSPANGVEGVRVDQDDDQDSSSLKLSQNIAVQTDFKTADSEVNTDQDIEKNLDKMMTERTLLKERYQEVLDKQRQVENQLQVQLKQLQQRREEEMKNHQEILKAIQDVTIKREETKKKIEKEKKEFLQKEQDLKAEIEKLCEKGRREVWEMELDRLKNQDGEINRNIMEETERAWKAEILSLESRKELLVLKLEEAEKEAELHLTYLKSTPPTLETVRSKQEWETRLNGVRIMKKNVRDQFNSHIQLVRNGAKLSSLPQIPTPTLPPPPSETDFMLQVFQPSPSLAPRMPFSIGQVTMPMVMPSADPRSLSFPILNPALSQPSQPSSPLPGSHGRNSPGLGSLVSPHGPHMPPAASIPPPPGLGGVKASAETPRPQPVDKLEKILEKLLTRFPQCNKAQMTNILQQIKTARTTMAGLTMEELIQLVAARLAEHERVAASTQPLGRIRALFPAPLAQISTPMFLPSAQVSYPGRSSHAPATCKLCLMCQKLVQPSELHPMACTHVLHKECIKFWAQTNTNDTCPFCPTLK",
# RN215_HUMAN
"MGPAARPALRSPPPPPPPPPSPLLLLLPLLPLWLGLAGPGAAADGSEPAAGAGRGGARAVRVDVRLPRQDALVLEGVRIGSEADPAPLLGGRLLLMDIVDAEQEAPVEGWIAVAYVGKEQAAQFHQENKGSGPQAYPKALVQQMRRALFLGASALLLLILNHNVVRELDISQLLLRPVIVLHYSSNVTKLLDALLQRTQATAEITSGESLSANIEWKLTLWTTCGLSKDGYGGWQDLVCLGGSRAQEQKPLQQLWNAILLVAMLLCTGLVVQAQRQASRQSQRELGGQVDLFKRRVVRRLASLKTRRCRLSRAAQGLPDPGAETCAVCLDYFCNKQWLRVLPCKHEFHRDCVDPWLMLQQTCPLCKFNVLGNRYSDD",
# RN216_HUMAN
"MEEGNNNEEVIHLNNFHCHRGQEWINLRDGPITISDSSDEERIPMLVTPAPQQHEEEDLDDDVILTEDDSEDDYGEFLDLGPPGISEFTKPSGQTEREPKPGPSHNQAANDIVNPRSEQKVIILEEGSLLYTESDPLETQNQSSEDSETELLSNLGESAALADDQAIEEDCWLDHPYFQSLNQQPREITNQVVPQERQPEAELGRLLFQHEFPGPAFPRPEPQQGGISGPSSPQPAHPLGEFEDQQLASDDEEPGPAFPMQESQEPNLENIWGQEAAEVDQELVELLVKETEARFPDVANGFIEEIIHFKNYYDLNVLCNFLLENPDYPKREDRIIINPSSSLLASQDETKLPKIDFFDYSKLTPLDQRCFIQAADLLMADFKVLSSQDIKWALHELKGHYAITRKALSDAIKKWQELSPETSGKRKKRKQMNQYSYIDFKFEQGDIKIEKRMFFLENKRRHCRSYDRRALLPAVQQEQEFYEQKIKEMAEHEDFLLALQMNEEQYQKDGQLIECRCCYGEFPFEELTQCADAHLFCKECLIRYAQEAVFGSGKLELSCMEGSCTCSFPTSELEKVLPQTILYKYYERKAEEEVAAAYADELVRCPSCSFPALLDSDVKRFSCPNPHCRKETCRKCQGLWKEHNGLTCEELAEKDDIKYRTSIEEKMTAARIRKCHKCGTGLIKSEGCNRMSCRCGAQMCYLCRVSINGYDHFCQHPRSPGAPCQECSRCSLWTDPTEDDEKLIEEIQKEAEEEQKRKNGENTFKRIGPPLEKPVEKVQRVEALPRPVPQNLPQPQMPPYAFAHPPFPLPPVRPVFNNFPLNMGPIPAPYVPPLPNVRVNYDFGPIHMPLEHNLPMHFGPQPRHRF",
# RN217_HUMAN
"MGEEQSTVSGGGGPQESQTLASGTAGHPEPPRPQGDSARAPPLRAASAEPSGGGCGSDWGCADTSAPEPARSLGPPGWSKSRAPAQPAGLALTGPLNPQTLPLQLELEEEEEEAGDRKEGGDEQQEAPPGEELEPRTRVGAADGLVLDVLGQRRPSLAKRQVFCSVYCVESDLPEAPASEQLSPPASPPGAPPVLNPPSTRSSFPSPRLSLPTDSLSPDGGSIELEFYLAPEPFSMPSLLGAPPYSGLGGVGDPYVPLMVLMCRVCLEDKPIKPLPCCKKAVCEECLKVYLSAQVQLGQVEIKCPITECFEFLEETTVVYNLTHEDSIKYKYFLELGRIDSSTKPCPQCKHFTTFKKKGHIPTPSRSESKYKIQCPTCQFVWCFKCHSPWHEGVNCKEYKKGDKLLRHWASEIEHGQRNAQKCPKCKIHIQRTEGCDHMTCSQCNTNFCYRCGERYRQLRFFGDHTSNLSIFGCKYRYLPERPHLRRLVRGSVCAGKLFIAPLIMVLGLALGAIAVVIGLFVFPIYCLCKKQRKRSRTGMHW",
# OBI1_HUMAN
"MAQTVQNVTLSLTLPITCHICLGKVRQPVICINNHVFCSICIDLWLKNNSQCPACRVPITPENPCKEIIGGTSESEPMLSHTVRKHLRKTRLELLHKEYEDEIDCLQKEVEELKSKNLSLESQIKTILDPLTLVQGNQNEDKHLVTDNPSKINPETVAEWKKKLRTANEIYEKVKDDVDKLKEANKKLKLENGGLVRENLRLKAEVDNRSPQKFGRFAVAALQSKVEQYERETNRLKKALERSDKYIEELESQVAQLKNSSEEKEAMNSICQTALSADGKGSKGSEEDVVSKNQGDSARKQPGSSTSSSSHLAKPSSSRLCDTSSARQESTSKADLNCSKNKDLYQEQVEVMLDVTDTSMDTYLEREWGNKPSDCVPYKDEELYDLPAPCTPLSLSCLQLSTPENRESSVVQAGGSKKHSNHLRKLVFDDFCDSSNVSNKDSSEDDISRSENEKKSECFSSPKTGFWDCCSTSYAQNLDFESSEGNTIANSVGEISSKLSEKSGLCLSKRLNSIRSFEMNRTRTSSEASMDAAYLDKISELDSMMSESDNSKSPCNNGFKSLDLDGLSKSSQGSEFLEEPDKLEEKTELNLSKGSLTNDQLENGSEWKPTSFFLLSPSDQEMNEDFSLHSSSCPVTNEIKPPSCLFQTEFSQGILLSSSHRLFEDQRFGSSLFKMSSEMHSLHNHLQSPWSTSFVPEKRNKNVNQSTKRKIQSSLSSASPSKATKS",
# RN220_HUMAN
"MDLHRAAFKMENSSYLPNPLASPALMVLASTAEASRDASIPCQQPRPFGVPVSVDKDVHIPFTNGSYTFASMYHRQGGVPGTFANRDFPPSLLHLHPQFAPPNLDCTPISMLNHSGVGAFRPFASTEDRESYQSAFTPAKRLKNCHDTESPHLRFSDADGKEYDFGTQLPSSSPGSLKVDDTGKKIFAVSGLISDREASSSPEDRNDRCKKKAAALFDSQAPICPICQVLLRPSELQEHMEQELEQLAQLPSSKNSLLKDAMAPGTPKSLLLSASIKREGESPTASPHSSATDDLHHSDRYQTFLRVRANRQTRLNARIGKMKRRKQDEGQREGSCMAEDDAVDIEHENNNRFEEYEWCGQKRIRATTLLEGGFRGSGFIMCSGKENPDSDADLDVDGDDTLEYGKPQYTEADVIPCTGEEPGEAKEREALRGAVLNGGPPSTRITPEFSKWASDEMPSTSNGESSKQEAMQKTCKNSDIEKITEDSAVTTFEALKARVRELERQLSRGDRYKCLICMDSYSMPLTSIQCWHVHCEECWLRTLGAKKLCPQCNTITAPGDLRRIYL",
# RN222_HUMAN
"MSEGESKDSSGSECPVCYEKFRDLEGASRTLSCGHVFCHDCLVKYLLSTRVDGQVQRTLVCPICRYVTFLSKKSSRWPSMLDKSSQTLAVPVGLPSVPPLDSLGHTNPLAASSPAWRPPPGQARPPGSPGQSAQLPLDLLPSLPRESQIFVISRHGMPLGEQDSVLPRRSLAELSEASLAPRSARAFCCRSRALLLITLIAVVAVVAAILPWVLLVRKQA",
# RN223_HUMAN
"MSSGQQVWHTAVPPPRRSSSIASMPRSPSSAGSPRSPGTPGSERVASPLECSICFSGYDNIFKTPKELSCTHVFCLECLARLAAAQPVGRPGGEAVPCPFCRQPTAVPPAGAPALCTSRQLQARMPAHLRREEPVWLEGTKLCCQPLPTTPGREPGFVCVDVGLSKPAEPPAPARDPAPRRGRLARCWARCRDWRRMALVSALLLMLFCVALWPVQCALKTGNLRCLPLPPRPPATSTAASPLGPLTDN",
# RN224_HUMAN
"MQDAAAGGPPGLGGGGPPEERTDCIICCSAYDLSGHLPRRLYCGHTFCQACVRRLDTPAPEQRWIPCPQCRQSTPTPRGGVAMLDLDLAAFLAVKAEREPARLEPLPLTSLKGSAITRQPAGLCPALGPQPHFPQPRYCCWGCGSLCCPPLGSPEV",
# RN225_HUMAN
"MPCPRPFWLRHSRAPQGSGPSSPGSLSAPRSPSRGEDQEEEEEEEGDGSPGSGPILPPASPVECLICVSSFDGVFKLPKRLDCGHVFCLECLARLSLATAGGGNAVACPVCRAPTRLAPRRGLPALPTQSGLLPRDARAPPSRQGSVRFDRRRGLLYLRPPPPPPGPRKARAPPPPPPLRLGRPLSRRLSLASPAWVFNAAVALAVLVAAGLVVSGVYIFFLIPHATSSGPPRPQLVALAPAPGFSWFPPRPPPGSPWAPAWTPRPTGPDLDTALPGTAEDALEPEAGPEDPAEAERTLDRRSDGTWGTEAGPGWAPWPRGARRLWGSQ",
# RNF24_HUMAN
"MSSDFPHYNFRMPNIGFQNLPLNIYIVVFGTAIFVFILSLLFCCYLIRLRHQAHKEFYAYKQVILKEKVKELNLHELCAVCLEDFKPRDELGICPCKHAFHRKCLIKWLEVRKVCPLCNMPVLQLAQLHSKQDRGPPQGPLPGAENIV",
# RNF25_HUMAN
"MAASASAAAGEEDWVLPSEVEVLESIYLDELQVIKGNGRTSPWEIYITLHPATAEDQDSQYVCFTLVLQVPAEYPHEVPQISIRNPRGLSDEQIHTILQVLGHVAKAGLGTAMLYELIEKGKEILTDNNIPHGQCVICLYGFQEKEAFTKTPCYHYFHCHCLARYIQHMEQELKAQGQEQEQERQHATTKQKAVGVQCPVCREPLVYDLASLKAAPEPQQPMELYQPSAESLRQQEERKRLYQRQQERGGIIDLEAERNRYFISLQQPPAPAEPESAVDVSKGSQPPSTLAAELSTSPAVQSTLPPPLPVATQHICEKIPGTRSNQQRLGETQKAMLDPPKPSRGPWRQPERRHPKGGECHAPKGTRDTQELPPPEGPLKEPMDLKPEPHSQGVEGPPQEKGPGSWQGPPPRRTRDCVRWERSKGRTPGSSYPRLPRGQGAYRPGTRRESLGLESKDGS",
# RNF26_HUMAN
"MEAVYLVVNGLGLVLDVLTLVLDLNFLLVSSLLASLAWLLAFVYNLPHTVLTSLLHLGRGVLLSLLALIEAVVRFTCGGLQALCTLLYSCCSGLESLKLLGHLASHGALRSREILHRGVLNVVSSGHALLRQACDICAIAMSLVAYVINSLVNICLIGTQNLFSLVLALWDAVTGPLWRMTDVVAAFLAHISSSAVAMAILLWTPCQLALELLASAARLLASFVLVNLTGLVLLACVLAVTVTVLHPDFTLRLATQALSQLHARPSYHRLREDVMRLSRLALGSEAWRRVWSRSLQLASWPNRGGAPGAPQGDPMRVFSVRTRRQDTLPEAGRRSEAEEEEARTIRVTPVRGRERLNEEEPPGGQDPWKLLKEQEERKKCVICQDQSKTVLLLPCRHLCLCQACTEILMRHPVYHRNCPLCRRGILQTLNVYL",
# RNF31_HUMAN
"MPGEEEERAFLVAREELASALRRDSGQAFSLEQLRPLLASSLPLAARYLQLDAARLVRCNAHGEPRNYLNTLSTALNILEKYGRNLLSPQRPRYWRGVKFNNPVFRSTVDAVQGGRDVLRLYGYTEEQPDGLSFPEGQEEPDEHQVATVTLEVLLLRTELSLLLQNTHPRQQALEQLLEDKVEDDMLQLSEFDPLLREIAPGPLTTPSVPGSTPGPCFLCGSAPGTLHCPSCKQALCPACDHLFHGHPSRAHHLRQTLPGVLQGTHLSPSLPASAQPRPQSTSLLALGDSSLSSPNPASAHLPWHCAACAMLNEPWAVLCVACDRPRGCKGLGLGTEGPQGTGGLEPDLARGRWACQSCTFENEAAAVLCSICERPRLAQPPSLVVDSRDAGICLQPLQQGDALLASAQSQVWYCIHCTFCNSSPGWVCVMCNRTSSPIPAQHAPRPYASSLEKGPPKPGPPRRLSAPLPSSCGDPEKQRQDKMREEGLQLVSMIREGEAAGACPEEIFSALQYSGTEVPLQWLRSELPYVLEMVAELAGQQDPGLGAFSCQEARRAWLDRHGNLDEAVEECVRTRRRKVQELQSLGFGPEEGSLQALFQHGGDVSRALTELQRQRLEPFRQRLWDSGPEPTPSWDGPDKQSLVRRLLAVYALPSWGRAELALSLLQETPRNYELGDVVEAVRHSQDRAFLRRLLAQECAVCGWALPHNRMQALTSCECTICPDCFRQHFTIALKEKHITDMVCPACGRPDLTDDTQLLSYFSTLDIQLRESLEPDAYALFHKKLTEGVLMRDPKFLWCAQCSFGFIYEREQLEATCPQCHQTFCVRCKRQWEEQHRGRSCEDFQNWKRMNDPEYQAQGLAMYLQENGIDCPKCKFSYALARGGCMHFHCTQCRHQFCSGCYNAFYAKNKCPEPNCRVKKSLHGHHPRDCLFYLRDWTALRLQKLLQDNNVMFNTEPPAGARAVPGGGCRVIEQKEVPNGLRDEACGKETPAGYAGLCQAHYKEYLVSLINAHSLDPATLYEVEELETATERYLHVRPQPLAGEDPPAYQARLLQKLTEEVPLGQSIPRRRK",
# RNF32_HUMAN
"MLKNKGHSSKKDNLAVNAVALQDHILHDLQLRNLSVADHSKTQVQKKENKSLKRDTKAIIDTGLKKTTQCPKLEDSEKEYVLDPKPPPLTLAQKLGLIGPPPPPLSSDEWEKVKQRSLLQGDSVQPCPICKEEFELRPQVLLSCSHVFHKACLQAFEKFTNKKTCPLCRKNQYQTRVIHDGARLFRIKCVTRIQAYWRGCVVRKWYRNLRKTVPPTDAKLRKKFFEKKFTEISHRILCSYNTNIEELFAEIDQCLAINRSVLQQLEEKCGHEITEEEWEKIQVQALRRETHECSICLAPLSAAGGQRVGAGRRSREMALLSCSHVFHHACLLALEEFSVGDRPPFHACPLCRSCYQKKILEC",
# RNF34_HUMAN
"MKAGATSMWASCCGLLNEVMGTGAVRGQQSAFAGATGPFRFTPNPEFSTYPPAATEGPNIVCKACGLSFSVFRKKHVCCDCKKDFCSVCSVLQENLRRCSTCHLLQETAFQRPQLMRLKVKDLRQYLILRNIPIDTCREKEDLVDLVLCHHGLGSEDDMDTSSLNSSRSQTSSFFTRSFFSNYTAPSATMSSFQGELMDGDQTSRSGVPAQVQSEITSANTEDDDDDDDEDDDDEEENAEDRNPGLSKERVRASLSDLSSLDDVEGMSVRQLKEILARNFVNYSGCCEKWELVEKVNRLYKENEENQKSYGERLQLQDEEDDSLCRICMDAVIDCVLLECGHMVTCTKCGKRMSECPICRQYVVRAVHVFKS",
# RNF38_HUMAN
"MACKISPGANSASLPGHPNKVICERVRLQSLFPLLPSDQNTTVQEDAHFKAFFQSEDSPSPKRQRLSHSVFDYTSASPAPSPPMRPWEMTSNRQPPSVRPSQHHFSGERCNTPARNRRSPPVRRQRGRRDRLSRHNSISQDENYHHLPYAQQQAIEEPRAFHPPNVSPRLLHPAAHPPQQNAVMVDIHDQLHQGTVPVSYTVTTVAPHGIPLCTGQHIPACSTQQVPGCSVVFSGQHLPVCSVPPPMLQACSVQHLPVPYAAFPPLISSDPFLIHPPHLSPHHPPHLPPPGQFVPFQTQQSRSPLQRIENEVELLGEHLPVGGFTYPPSAHPPTLPPSAPLQFLTHDPLHQEVSFGVPYPPFMPRRLTGRSRYRSQQPIPPPPYHPSLLPYVLSMLPVPPAVGPTFSFELDVEDGEVENYEALLNLAERLGEAKPRGLTKADIEQLPSYRFNPNNHQSEQTLCVVCMCDFESRQLLRVLPCNHEFHAKCVDKWLKANRTCPICRADASEVHRDSE",
# RNF39_HUMAN
"MWWRDLTRLRLWLKREAIPGEGRKAAKVNAGVGEKGIYTASSRGGPPSARSKAVTVVAEGAASRSWLSMDAPELGPGLVERLEQLATCPLCGGSFEDPVLLACEHSFCRACLARRWGTPPATGTEASPTACPCCGLPCPRRSLRSNVRLAVEVRISRELREKLAEPGARAGRRRGGRIPTMGCLDLPGEDMRKTWRRFEVPTSKSSNSEDDLPEDYPVVKKMLHRLTADLTLDPGTAHRRLLISADRRSVQLAPPGTPAPPDGPKRFDQLPAVLGAQGFGAGRHCWEVETADAASCRDSSGEDADDEESHYAVGAAGESVQRKGCVRLCPAGAVWAVEGRGGRLWALTAPEPTLLGGVEPPPRRIRVDLDWERGRVAFYDGRSLDLLYAFQAPGPLGERIFPLFCTCDPRAPLRIVPAES",
# RNF4_HUMAN
"MSTRKRRGGAINSRQAQKRTREATSTPEISLEAEPIELVETAGDEIVDLTCESLEPVVVDLTHNDSVVIVDERRRPRRNARRLPQDHADSCVVSSDDEELSRDRDVYVTTHTPRNARDEGATGLRPSGTVSCPICMDGYSEIVQNGRLIVSTECGHVFCSQCLRDSLKNANTCPTCRKKINHKRYHPIYI",
# BRE1B_HUMAN
"MSGPGNKRAAGDGGSGPPEKKLSREEKTTTTLIEPIRLGGISSTEEMDLKVLQFKNKKLAERLEQRQACEDELRERIEKLEKRQATDDATLLIVNRYWAQLDETVEALLRCHESQGELSSAPEAPGTQEGPTCDGTPLPEPGTSELRDPLLMQLRPPLSEPALAFVVALGASSSEEVELELQGRMEFSKAAVSRVVEASDRLQRRVEELCQRVYSRGDSEPLSEAAQAHTRELGRENRRLQDLATQLQEKHHRISLEYSELQDKVTSAETKVLEMETTVEDLQWDIEKLRKREQKLNKHLAEALEQLNSGYYVSGSSSGFQGGQITLSMQKFEMLNAELEENQELANSRMAELEKLQAELQGAVRTNERLKVALRSLPEEVVRETGEYRMLQAQFSLLYNESLQVKTQLDEARGLLLATKNSHLRHIEHMESDELGLQKKLRTEVIQLEDTLAQVRKEYEMLRIEFEQNLAANEQAGPINREMRHLISSLQNHNHQLKGDAQRYKRKLREVQAEIGKLRAQASGSAHSTPNLGHPEDSGVSAPAPGKEEGGPGPVSTPDNRKEMAPVPGTTTTTTSVKKEELVPSEEDFQGITPGAQGPSSRGREPEARPKRELREREGPSLGPPPVASALSRADREKAKVEETKRKESELLKGLRAELKKAQESQKEMKLLLDMYKSAPKEQRDKVQLMAAERKAKAEVDELRSRIRELEERDRRESKKIADEDALRRIRQAEEQIEHLQRKLGATKQEEEALLSEMDVTGQAFEDMQEQNGRLLQQLREKDDANFKLMSERIKANQIHKLLREEKDELGEQVLGLKSQVDAQLLTVQKLEEKERALQGSLGGVEKELTLRSQALELNKRKAVEAAQLAEDLKVQLEHVQTRLREIQPCLAESRAAREKESFNLKRAQEDISRLRRKLEKQRKVEVYADADEILQEEIKEYKARLTCPCCNTRKKDAVLTKCFHVFCFECVRGRYEARQRKCPKCNAAFGAHDFHRIYIS",
# RNF41_HUMAN
"MGYDVTRFQGDVDEDLICPICSGVLEEPVQAPHCEHAFCNACITQWFSQQQTCPVDRSVVTVAHLRPVPRIMRNMLSKLQIACDNAVFGCSAVVRLDNLMSHLSDCEHNPKRPVTCEQGCGLEMPKDELPNHNCIKHLRSVVQQQQTRIAELEKTSAEHKHQLAEQKRDIQLLKAYMRAIRSVNPNLQNLEETIEYNEILEWVNSLQPARVTRWGGMISTPDAVLQAVIKRSLVESGCPASIVNELIENAHERSWPQGLATLETRQMNRRYYENYVAKRIPGKQAVVVMACENQHMGDDMVQEPGLVMIFAHGVEEI",
# RNF43_HUMAN
"MSGGHQLQLAALWPWLLMATLQAGFGRTGLVLAAAVESERSAEQKAIIRVIPLKMDPTGKLNLTLEGVFAGVAEITPAEGKLMQSHPLYLCNASDDDNLEPGFISIVKLESPRRAPRPCLSLASKARMAGERGASAVLFDITEDRAAAEQLQQPLGLTWPVVLIWGNDAEKLMEFVYKNQKAHVRIELKEPPAWPDYDVWILMTVVGTIFVIILASVLRIRCRPRHSRPDPLQQRTAWAISQLATRRYQASCRQARGEWPDSGSSCSSAPVCAICLEEFSEGQELRVISCLHEFHRNCVDPWLHQHRTCPLCMFNITEGDSFSQSLGPSRSYQEPGRRLHLIRQHPGHAHYHLPAAYLLGPSRSAVARPPRPGPFLPSQEPGMGPRHHRFPRAAHPRAPGEQQRLAGAQHPYAQGWGLSHLQSTSQHPAACPVPLRRARPPDSSGSGESYCTERSGYLADGPASDSSSGPCHGSSSDSVVNCTDISLQGVHGSSSTFCSSLSSDFDPLVYCSPKGDPQRVDMQPSVTSRPRSLDSVVPTGETQVSSHVHYHRHRHHHYKKRFQWHGRKPGPETGVPQSRPPIPRTQPQPEPPSPDQQVTRSNSAAPSGRLSNPQCPRALPEPAPGPVDASSICPSTSSLFNLQKSSLSARHPQRKRRGGPSEPTPGSRPQDATVHPACQIFPHYTPSVAYPWSPEAHPLICGPPGLDKRLLPETPGPCYSNSQPVWLCLTPRQPLEPHPPGEGPSEWSSDTAEGRPCPYPHCQVLSAQPGSEEELEELCEQAV",
# RNF44_HUMAN
"MRPWALAVTRWPPSAPVGQRRFSAGPGSTPGQLWGSPGLEGPLASPPARDERLPSQQPPSRPPHLPVEERRASAPAGGSPRMLHPATQQSPFMVDLHEQVHQGPVPLSYTVTTVTTQGFPLPTGQHIPGCSAQQLPACSVMFSGQHYPLCCLPPPLIQACTMQQLPVPYQAYPHLISSDHYILHPPPPAPPPQPTHMAPLGQFVSLQTQHPRMPLQRLDNDVDLRGDQPSLGSFTYSTSAPGPALSPSVPLHYLPHDPLHQELSFGVPYSHMMPRRLSTQRYRLQQPLPPPPPPPPPPPYYPSFLPYFLSMLPMSPTAMGPTISLDLDVDDVEMENYEALLNLAERLGDAKPRGLTKADIEQLPSYRFNPDSHQSEQTLCVVCFSDFEARQLLRVLPCNHEFHTKCVDKWLKANRTCPICRADASEVPREAE",
# RNF5_HUMAN
"MAAAEEEDGGPEGPNRERGGAGATFECNICLETAREAVVSVCGHLYCWPCLHQWLETRPERQECPVCKAGISREKVVPLYGRGSQKPQDPRLKTPPRPQGQRPAPESRGGFQPFGDTGGFHFSFGVGAFPFGFFTTVFNAHEPFRRGTGVDLGQGHPASSWQDSLFLFLAIFFFFWLLSI",
# RNF6_HUMAN
"MNQSRSRSDGGSEETLPQDHNHHENERRWQQERLHREEAYYQFINELNDEDYRLMRDHNLLGTPGEITSEELQQRLDGVKEQLASQPDLRDGTNYRDSEVPRESSHEDSLLEWLNTFRRTGNATRSGQNGNQTWRAVSRTNPNNGEFRFSLEIHVNHENRGFEIHGEDYTDIPLSDSNRDHTANRQQRSTSPVARRTRSQTSVNFNGSSSNIPRTRLASRGQNPAEGSFSTLGRLRNGIGGAAGIPRANASRTNFSSHTNQSGGSELRQREGQRFGAAHVWENGARSNVTVRNTNQRLEPIRLRSTSNSRSRSPIQRQSGTVYHNSQRESRPVQQTTRRSVRRRGRTRVFLEQDRERERRGTAYTPFSNSRLVSRITVEEGEESSRSSTAVRRHPTITLDLQVRRIRPGENRDRDSIANRTRSRVGLAENTVTIESNSGGFRRTISRLERSGIRTYVSTITVPLRRISENELVEPSSVALRSILRQIMTGFGELSSLMEADSESELQRNGQHLPDMHSELSNLGTDNNRSQHREGSSQDRQAQGDSTEMHGENETTQPHTRNSDSRGGRQLRNPNNLVETGTLPILRLAHFFLLNESDDDDRIRGLTKEQIDNLSTRHYEHNSIDSELGKICSVCISDYVTGNKLRQLPCMHEFHIHCIDRWLSENCTCPICRQPVLGSNIANNG",
# RBX2_HUMAN
"MADVEDGEETCALASHSGSSGSKSGGDKMFSLKKWNAVAMWSWDVECDTCAICRVQVMDACLRCQAENKQEDCVVVWGECNHSFHNCCMSLWVKQNNRCPLCQQDWVVQRIGK",
# RNF8_HUMAN
"MGEPGFFVTGDRAGGRSWCLRRVGMSAGWLLLEDGCEVTVGRGFGVTYQLVSKICPLMISRNHCVLKQNPEGQWTIMDNKSLNGVWLNRARLEPLRVYSIHQGDYIQLGVPLENKENAEYEYEVTEEDWETIYPCLSPKNDQMIEKNKELRTKRKFSLDELAGPGAEGPSNLKSKINKVSCESGQPVKSQGKGEVASTPSDNLDPKLTALEPSKTTGAPIYPGFPKVTEVHHEQKASNSSASQRSLQMFKVTMSRILRLKIQMQEKHEAVMNVKKQTQKGNSKKVVQMEQELQDLQSQLCAEQAQQQARVEQLEKTFQEEEQHLQGLEIAQGEKDLKQQLAQALQEHWALMEELNRSKKDFEAIIQAKNKELEQTKEEKEKMQAQKEEVLSHMNDVLENELQCIICSEYFIEAVTLNCAHSFCSYCINEWMKRKIECPICRKDIKSKTYSLVLDNCINKMVNNLSSEVKERRIVLIRERKAKRLF",
# RNFT1_HUMAN
"MPLFLLSLPTPPSASGHERRQRPEAKTSGSEKKYLRAMQANRSQLHSPPGTGSSEDASTPQCVHTRLTGEGSCPHSGDVHIQINSIPKECAENASSRNIRSGVHSCAHGCVHSRLRGHSHSEARLTDDTAAESGDHGSSSFSEFRYLFKWLQKSLPYILILSVKLVMQHITGISLGIGLLTTFMYANKSIVNQVFLRERSSKIQCAWLLVFLAGSSVLLYYTFHSQSLYYSLIFLNPTLDHLSFWEVFWIVGITDFILKFFFMGLKCLILLVPSFIMPFKSKGYWYMLLEELCQYYRTFVPIPVWFRYLISYGEFGNVTRWSLGILLALLYLILKLLEFFGHLRTFRQVLRIFFTQPSYGVAASKRQCSDVDDICSICQAEFQKPILLICQHIFCEECMTLWFNREKTCPLCRTVISDHINKWKDGATSSHLQIY",
# RNFT2_HUMAN
"MWLFTVNQVLRKMQRRHSSNTDNIPPERNRSQALSSEASVDEGGVFESLKAEAASPPALFSGLSGSLPTSSFPSSLVLGSSAGGGDVFIQMPASREEGGGRGEGGAYHHRQPHHHFHHGGHRGGSLLQHVGGDHRGHSEEGGDEQPGTPAPALSELKAVICWLQKGLPFILILLAKLCFQHKLGIAVCIGMASTFAYANSTLREQVSLKEKRSVLVILWILAFLAGNTLYVLYTFSSQQLYNSLIFLKPNLEMLDFFDLLWIVGIADFVLKYITIALKCLIVALPKIILAVKSKGKFYLVIEELSQLFRSLVPIQLWYKYIMGDDSSNSYFLGGVLIVLYSLCKSFDICGRVGGVRKALKLLCTSQNYGVRATGQQCTEAGDICAICQAEFREPLILLCQHVFCEECLCLWLDRERTCPLCRSVAVDTLRCWKDGATSAHFQVY",
# RSPRY_HUMAN
"MIVFGWAVFLASRSLGQGLLLTLEEHIAHFLGTGGAATTMGNSCICRDDSGTDDSVDTQQQQAENSAVPTADTRSQPRDPVRPPRRGRGPHEPRRKKQNVDGLVLDTLAVIRTLVDNDQEPPYSMITLHEMAETDEGWLDVVQSLIRVIPLEDPLGPAVITLLLDECPLPTKDALQKLTEILNLNGEVACQDSSHPAKHRNTSAVLGCLAEKLAGPASIGLLSPGILEYLLQCLKLQSHPTVMLFALIALEKFAQTSENKLTISESSISDRLVTLESWANDPDYLKRQVGFCAQWSLDNLFLKEGRQLTYEKVNLSSIRAMLNSNDVSEYLKISPHGLEARCDASSFESVRCTFCVDAGVWYYEVTVVTSGVMQIGWATRDSKFLNHEGYGIGDDEYSCAYDGCRQLIWYNARSKPHIHPCWKEGDTVGFLLDLNEKQMIFFLNGNQLPPEKQVFSSTVSGFFAAASFMSYQQCEFNFGAKPFKYPPSMKFSTFNDYAFLTAEEKIILPRHRRLALLKQVSIRENCCSLCCDEVADTQLKPCGHSDLCMDCALQLETCPLCRKEIVSRIRQISHIS",
# SCAFB_HUMAN
"MKKKTVCTLNMGDKKYEDMEGEENGDNTISTGLLYSEADRCPICLNCLLEKEVGFPESCNHVFCMTCILKWAETLASCPIDRKPFQAVFKFSALEGYVKVQVKKQLRETKDKKNENSFEKQVSCHENSKSCIRRKAIVREDLLSAKVCDLKWIHRNSLYSETGGKKNAAIKINKPQRSNWSTNQCFRNFFSNMFSSVSHSGESSFTYRAYCTEFIEASEISALIRQKRHELELSWFPDTLPGIGRIGFIPWNVETEVLPLISSVLPRTIFPTSTISFEHFGTSCKGYALAHTQEGEEKKQTSGTSNTRGSRRKPAMTTPTRRSTRNTRAETASQSQRSPISDNSGCDAPGNSNPSLSVPSSAESEKQTRQAPKRKSVRRGRKPPLLKKKLRSSVAAPEKSSSNDSVDEETAESDTSPVLEKEHQPDVDSSNICTVQTHVENQSANCLKSCNEQIEESEKHTANYDTEERVGSSSSESCAQDLPVLVGEEGEVKKLENTGIEANVLCLESEISENILEKGGDPLEKQDQISGLSQSEVKTDVCTVHLPNDFPTCLTSESKVYQPVSCPLSDLSENVESVVNEEKITESSLVEITEHKDFTLKTEELIESPKLESSEGEIIQTVDRQSVKSPEVQLLGHVETEDVEIIATCDTFGNEDFNNIQDSENNLLKNNLLNTKLEKSLEEKNESLTEHPRSTELPKTHIEQIQKHFSEDNNEMIPMECDSFCSDQNESEVEPSVNADLKQMNENSVTHCSENNMPSSDLADEKVETVSQPSESPKDTIDKTKKPRTRRSRFHSPSTTWSPNKDTPQEKKRPQSPSPRRETGKESRKSQSPSPKNESARGRKKSRSQSPKKDIARERRQSQSRSPKRDTTRESRRSESLSPRRETSRENKRSQPRVKDSSPGEKSRSQSRERESDRDGQRRERERRTRKWSRSRSHSRSPSRCRTKSKSSSFGRIDRDSYSPRWKGRWANDGWRCPRGNDRYRKNDPEKQNENTRKEKNDIHLDADDPNSADKHRNDCPNWITEKINSGPDPRTRNPEKLKESHWEENRNENSGNSWNKNFGSGWVSNRGRGRGNRGRGTYRSSFAYKDQNENRWQNRKPLSGNSNSSGSESFKFVEQQSYKRKSEQEFSFDTPADRSGWTSASSWAVRKTLPADVQNYYSRRGRNSSGPQSGWMKQEEETSGQDSSLKDQTNQQVDGSQLPINMMQPQMNVMQQQMNAQHQPMNIFPYPVGVHAPLMNIQRNPFNIHPQLPLHLHTGVPLMQVATPTSVSQGLPPPPPPPPPSQQVNYIASQPDGKQLQGIPSSSHVSNNMSTPVLPAPTAAPGNTGMVQGPSSGNTSSSSHSKASNAAVKLAESKVSVAVEASADSSKTDKKLQIQEKAAQEVKLAIKPFYQNKDITKEEYKEIVRKAVDKVCHSKSGEVNSTKVANLVKAYVDKYKYSRKGSQKKTLEEPVSTEKNIG",
# SH3R1_HUMAN
"MDESALLDLLECPVCLERLDASAKVLPCQHTFCKRCLLGIVGSRNELRCPECRTLVGSGVEELPSNILLVRLLDGIKQRPWKPGPGGGSGTNCTNALRSQSSTVANCSSKDLQSSQGGQQPRVQSWSPPVRGIPQLPCAKALYNYEGKEPGDLKFSKGDIIILRRQVDENWYHGEVNGIHGFFPTNFVQIIKPLPQPPPQCKALYDFEVKDKEADKDCLPFAKDDVLTVIRRVDENWAEGMLADKIGIFPISYVEFNSAAKQLIEWDKPPVPGVDAGECSSAAAQSSTAPKHSDTKKNTKKRHSFTSLTMANKSSQASQNRHSMEISPPVLISSSNPTAAARISELSGLSCSAPSQVHISTTGLIVTPPPSSPVTTGPSFTFPSDVPYQAALGTLNPPLPPPPLLAATVLASTPPGATAAAAAAGMGPRPMAGSTDQIAHLRPQTRPSVYVAIYPYTPRKEDELELRKGEMFLVFERCQDGWFKGTSMHTSKIGVFPGNYVAPVTRAVTNASQAKVPMSTAGQTSRGVTMVSPSTAGGPAQKLQGNGVAGSPSVVPAAVVSAAHIQTSPQAKVLLHMTGQMTVNQARNAVRTVAAHNQERPTAAVTPIQVQNAAGLSPASVGLSHHSLASPQPAPLMPGSATHTAAISISRASAPLACAAAAPLTSPSITSASLEAEPSGRIVTVLPGLPTSPDSASSACGNSSATKPDKDSKKEKKGLLKLLSGASTKRKPRVSPPASPTLEVELGSAELPLQGAVGPELPPGGGHGRAGSCPVDGDGPVTTAVAGAALAQDAFHRKASSLDSAVPIAPPPRQACSSLGPVLNESRPVVCERHRVVVSYPPQSEAELELKEGDIVFVHKKREDGWFKGTLQRNGKTGLFPGSFVENI",
# SH3R2_HUMAN
"MDDLTLLDLLECPVCFEKLDVTAKVLPCQHTFCKPCLQRVFKAHKELRCPECRTPVFSNIEALPANLLLVRLLDGVRSGQSSGRGGSFRRPGTMTLQDGRKSRTNPRRLQASPFRLVPNVRIHMDGVPRAKALCNYRGQNPGDLRFNKGDIILLRRQLDENWYQGEINGISGNFPASSVEVIKQLPQPPPLCRALYNFDLRGKDKSENQDCLTFLKDDIITVISRVDENWAEGKLGDKVGIFPILFVEPNLTARHLLEKNKGRQSSRTKNLSLVSSSSRGNTSTLRRGPGSRRKVPGQFSITTALNTLNRMVHSPSGRHMVEISTPVLISSSNPSVITQPMEKADVPSSCVGQVSTYHPAPVSPGHSTAVVSLPGSQQHLSANMFVALHSYSAHGPDELDLQKGEGVRVLGKCQDGWLRGVSLVTGRVGIFPNNYVIPIFRKTSSFPDSRSPGLYTTWTLSTSSVSSQGSISEGDPRQSRPFKSVFVPTAIVNPVRSTAGPGTLGQGSLRKGRSSMRKNGSLQRPLQSGIPTLVVGSLRRSPTMVLRPQQFQFYQPQGIPSSPSAVVVEMGSKPALTGEPALTCISRGSEAWIHSAASSLIMEDKEIPIKSEPLPKPPASAPPSILVKPENSRNGIEKQVKTVRFQNYSPPPTKHYTSHPTSGKPEQPATLKASQPEAASLGPEMTVLFAHRSGCHSGQQTDLRRKSALGKATTLVSTASGTQTVFPSK",
# SH3R3_HUMAN
"MLLGASWLCASKAAAAAAQSEGDEDRPGERRRRRAAATAAGAGEDMDESSLLDLLECSVCLERLDTTAKVLPCQHTFCRRCLESIVCSRHELRCPECRILVGCGVDELPANILLVRLLDGIRQRPRAGTSPGGSPPARPIPGQSAAPTLAGGGGGAAGSTPGSPVFLSAAAGSTAGSLRELATSRTAPAAKNPCLLPYGKALYSYEGKEPGDLKFNKGDIIVLRRKVDEQWYHGELHGTQGFLPASYIQCIQPLPHAPPQGKALYDFEMKDKDQDKDCLTFTKDEILTVLRRVDENWAEGMLGDKIGIFPLLYVELNDSAKQLIEMDKPCPAAASSCNASLPSDSGAVASVAPSPTLSSSGAVSAFQRRVDGKKNTKKRHSFTALSVTHRSSQAASHRHSMEISAPVLISSSDPRAAARIGDLAHLSCAAPTQDVSSSAGSTPTAVPRAASVSGEQGTPPKVQLPLNVYLALYAYKPQKSDELELHKGEMYRVLEKCQDGWFKGASLRTGVSGVFPGNYVTPVSRVPAGGAGPPRNNVVGGSPLAKGITTTMHPGSGSLSSLATATRPALPITTPQAHAQHPTASPPTGSCLRHSAQPTASQARSTISTAAHSAAQAQDRPTATVSPLRTQNSPSRLPATSLRPHSVVSPQHSHQPPVQMCPRPAIPLTSAASAITPPNVSAANLNGEAGGGPIGVLSTSSPTNTGCKLDEKKSEKKEKKSGLLKLLAGASTKKKSRSPPSVSPTHDPQVAVDALLQGAVGPEVSSLSIHGRAGSCPIESEMQGAMGMEPLHRKAGSLDLNFTSPSRQAPLSMAAIRPEPKLLPRERYRVVVSYPPQSEAEIELKEGDIVFVHKKREDGWYKGTLQRNGRTGLFPGSFVESF",
# SHPRH_HUMAN
"MSSRRKRAPPVRVDEEKRQQLHWNMHEDRRNEPIIISDDDEQPCPGSDTSSAHYIILSDSLKEEVAHRDKKRCSKVVSFSKPIEKEETVGIFSPLSVKLNIVISPYHFDNSWKAFLGELTLQLLPAQSLIENFSERSITLMSSESSNQFLIYVHSKGEDVEKQKKEPMSICDKGILVESSFSGEMLEDLGWLQKKRRIKLYQKPEGNHIIKVGIYLLEAGLAKLDFLSDANSRMKKFNQLMKKVMEKLHNSIIPDVLEEDEDDPESEPEGQDIDELYHFVKQTHQQETQSIQVDVQHPALIPVLRPYQREAVNWMLQQECFRSSPATESALHFLWREIVTSEGLKLYYNPYTGCIIREYPNSGPQLLGGILADEMGLGKTVEVLALILTHTRQDVKQDALTLPEGKVVNYFIPSHYFGGKLKKTEIQNIEFEPKEKVQCPPTRVMILTAVKEMNGKKGVSILSIYKYVSSIYRYDVQRNRSLLKRMLKCLIFEGLVKQIKGHGFSGTFTLGKNYKEEDICDKTKKQAVGSPRKIQKETRKSGNKDTDSEYLPSDTSDDDDDPYYYYYKSRRNRSKLRKKLVPSTKKGKSQPFINPDSQGHCPATSDSGITDVAMSKSTCISEFNQEHETEDCAESLNHADSDVPPSNTMSPFNTSDYRFECICGELDQIDRKPRVQCLKCHLWQHAKCVNYDEKNLKIKPFYCPHCLVAMEPVSTRATLIISPSSICHQWVDEINRHVRSSSLRVLVYQGVKKDGFLQPHFLAEQDIVIITYDVLRSELNYVDIPHSNSEDGRRLRNQKRYMAIPSPLVAVEWWRICLDEAQMVECPTVKAAEMAQRLSGINRWCISGTPVQRGLEDLFGLVVFLGIEPYCVKHWWVRLLYRPYCKKNPQHLYSFIAKILWRSAKKDVIDQIQIPPQTEEIHWLHFSPVERHFYHRQHEVCCQDVVVKLRKISDWALKLSSLDRRTVTSILYPLLRLRQACCHPQAVRGEFLPLQKSTMTMEELLTSLQKKCGTECEEAHRQLVCALNGLAGIHIIKGEYALAAELYREVLRSSEEHKGKLKTDSLQRLHATHNLMELLIARHPGIPPTLRDGRLEEEAKQLREHYMSKCNTEVAEAQQALYPVQQTIHELQRKIHSNSPWWLNVIHRAIEFTIDEELVQRVRNEITSNYKQQTGKLSMSEKFRDCRGLQFLLTTQMEELNKCQKLVREAVKNLEGPPSRNVIESATVCHLRPARLPLNCCVFCKADELFTEYESKLFSNTVKGQTAIFEEMIEDEEGLVDDRAPTTTRGLWAISETERSMKAILSFAKSHRFDVEFVDEGSTSMDLFEAWKKEYKLLHEYWMALRNRVSAVDELAMATERLRVRDPREPKPNPPVLHIIEPHEVEQNRIKLLNDKAVATSQLQKKLGQLLYLTNLEKSQDKTSGGVNPEPCPICARQLGKQWAVLTCGHCFCNECISIIIEQYSVGSHRSSIKCAICRQTTSHKEISYVFTSEKANQEEDIPVKGSHSTKVEAVVRTLMKIQLRDPGAKALVFSTWQDVLDIISKALTDNNMEFAQISRVKTFQENLSAFKRDPQINILLLPLHTGSNGLTIIEATHVLLVEPILNPAHELQAIGRVHRIGQTKPTIVHRFLIKATIEERMQAMLKTAERSHTNSSAKHSEASVLTVADLADLFTKETEELE",
# SIAH1_HUMAN
"MSRQTATALPTGTSKCPPSQRVPALTGTTASNNDLASLFECPVCFDYVLPPILQCQSGHLVCSNCRPKLTCCPTCRGPLGSIRNLAMEKVANSVLFPCKYASSGCEITLPHTEKADHEELCEFRPYSCPCPGASCKWQGSLDAVMPHLMHQHKSITTLQGEDIVFLATDINLPGAVDWVMMQSCFGFHFMLVLEKQEKYDGHQQFFAIVQLIGTRKQAENFAYRLELNGHRRRLTWEATPRSIHEGIATAIMNSDCLVFDTSIAQLFAENGNLGINVTISMC",
# SIAH2_HUMAN
"MSRPSSTGPSANKPCSKQPPPQPQHTPSPAAPPAAATISAAGPGSSAVPAAAAVISGPGGGGGAGPVSPQHHELTSLFECPVCFDYVLPPILQCQAGHLVCNQCRQKLSCCPTCRGALTPSIRNLAMEKVASAVLFPCKYATTGCSLTLHHTEKPEHEDICEYRPYSCPCPGASCKWQGSLEAVMSHLMHAHKSITTLQGEDIVFLATDINLPGAVDWVMMQSCFGHHFMLVLEKQEKYEGHQQFFAIVLLIGTRKQAENFAYRLELNGNRRRLTWEATPRSIHDGVAAAIMNSDCLVFDTAIAHLFADNGNLGINVTISTCCP",
# SIAH3_HUMAN
"MLFFTQCFGAVLDLIHLRFQHYKAKRVFSAAGQLVCVVNPTHNLKYVSSRRAVTQSAPEQGSFHPHHLSHHHCHHRHHHHLRHHAHPHHLHHQEAGLHANPVTPCLCMCPLFSCQWEGRLEVVVPHLRQIHRVDILQGAEIVFLATDMHLPAPADWIIMHSCLGHHFLLVLRKQERHEGHPQFFATMMLIGTPTQADCFTYRLELNRNHRRLKWEATPRSVLECVDSVITDGDCLVLNTSLAQLFSDNGSLAIGIAITATEVLPSEAEM",
# SMUF1_HUMAN
"MSNPGTRRNGSSIKIRLTVLCAKNLAKKDFFRLPDPFAKIVVDGSGQCHSTDTVKNTLDPKWNQHYDLYVGKTDSITISVWNHKKIHKKQGAGFLGCVRLLSNAISRLKDTGYQRLDLCKLNPSDTDAVRGQIVVSLQTRDRIGTGGSVVDCRGLLENEGTVYEDSGPGRPLSCFMEEPAPYTDSTGAAAGGGNCRFVESPSQDQRLQAQRLRNPDVRGSLQTPQNRPHGHQSPELPEGYEQRTTVQGQVYFLHTQTGVSTWHDPRIPSPSGTIPGGDAAFLYEFLLQGHTSEPRDLNSVNCDELGPLPPGWEVRSTVSGRIYFVDHNNRTTQFTDPRLHHIMNHQCQLKEPSQPLPLPSEGSLEDEELPAQRYERDLVQKLKVLRHELSLQQPQAGHCRIEVSREEIFEESYRQIMKMRPKDLKKRLMVKFRGEEGLDYGGVAREWLYLLCHEMLNPYYGLFQYSTDNIYMLQINPDSSINPDHLSYFHFVGRIMGLAVFHGHYINGGFTVPFYKQLLGKPIQLSDLESVDPELHKSLVWILENDITPVLDHTFCVEHNAFGRILQHELKPNGRNVPVTEENKKEYVRLYVNWRFMRGIEAQFLALQKGFNELIPQHLLKPFDQKELELIIGGLDKIDLNDWKSNTRLKHCVADSNIVRWFWQAVETFDEERRARLLQFVTGSTRVPLQGFKALQGSTGAAGPRLFTIHLIDANTDNLPKAHTCFNRIDIPPYESYEKLYEKLLTAVEETCGFAVE",
# SMUF2_HUMAN
"MSNPGGRRNGPVKLRLTVLCAKNLVKKDFFRLPDPFAKVVVDGSGQCHSTDTVKNTLDPKWNQHYDLYIGKSDSVTISVWNHKKIHKKQGAGFLGCVRLLSNAINRLKDTGYQRLDLCKLGPNDNDTVRGQIVVSLQSRDRIGTGGQVVDCSRLFDNDLPDGWEERRTASGRIQYLNHITRTTQWERPTRPASEYSSPGRPLSCFVDENTPISGTNGATCGQSSDPRLAERRVRSQRHRNYMSRTHLHTPPDLPEGYEQRTTQQGQVYFLHTQTGVSTWHDPRVPRDLSNINCEELGPLPPGWEIRNTATGRVYFVDHNNRTTQFTDPRLSANLHLVLNRQNQLKDQQQQQVVSLCPDDTECLTVPRYKRDLVQKLKILRQELSQQQPQAGHCRIEVSREEIFEESYRQVMKMRPKDLWKRLMIKFRGEEGLDYGGVAREWLYLLSHEMLNPYYGLFQYSRDDIYTLQINPDSAVNPEHLSYFHFVGRIMGMAVFHGHYIDGGFTLPFYKQLLGKSITLDDMELVDPDLHNSLVWILENDITGVLDHTFCVEHNAYGEIIQHELKPNGKSIPVNEENKKEYVRLYVNWRFLRGIEAQFLALQKGFNEVIPQHLLKTFDEKELELIICGLGKIDVNDWKVNTRLKHCTPDSNIVKWFWKAVEFFDEERRARLLQFVTGSSRVPLQGFKALQGAAGPRLFTIHQIDACTNNLPKAHTCFNRIDIPPYESYEKLYEKLLTAIEETCGFAVE",
# CHIP_HUMAN
"MKGKEEKEGGARLGAGGGSPEKSPSAQELKEQGNRLFVGRKYPEAAACYGRAITRNPLVAVYYTNRALCYLKMQQHEQALADCRRALELDGQSVKAHFFLGQCQLEMESYDEAIANLQRAYSLAKEQRLNFGDDIPSALRIAKKKRWNSIEERRIHQESELHSYLSRLIAAERERELEECQRNHEGDEDDSHVRAQQACIEAKHDKYMADMDELFSQVDEKRKKRDIPDYLCGKISFELMREPCITPSGITYDRKDIEEHLQRVGHFDPVTRSPLTQEQLIPNLAMKEVIDAFISENGWVEDY",
# SYVN1_HUMAN
"MFRTAVMMAASLALTGAVVAHAYYLKHQFYPTVVYLTKSSPSMAVLYIQAFVLVFLLGKVMGKVFFGQLRAAEMEHLLERSWYAVTETCLAFTVFRDDFSPRFVALFTLLLFLKCFHWLAEDRVDFMERSPNISWLFHCRIVSLMFLLGILDFLFVSHAYHSILTRGASVQLVFGFEYAILMTMVLTIFIKYVLHSVDLQSENPWDNKAVYMLYTELFTGFIKVLLYMAFMTIMIKVHTFPLFAIRPMYLAMRQFKKAVTDAIMSRRAIRNMNTLYPDATPEELQAMDNVCIICREEMVTGAKRLPCNHIFHTSCLRSWFQRQQTCPTCRMDVLRASLPAQSPPPPEPADQGPPPAPHPPPLLPQPPNFPQGLLPPFPPGMFPLWPPMGPFPPVPPPPSSGEAVAPPSTSAAALSRPSGAATTTAAGTSATAASATASGPGSGSAPEAGPAPGFPFPPPWMGMPLPPPFAFPPMPVPPAGFAGLTPEELRALEGHERQHLEARLQSLRNIHTLLDAAMLQINQYLTVLASLGPPRPATSVNSTEETATTVVAAASSTSIPSSEATTPTPGASPPAPEMERPPAPESVGTEEMPEDGEPDAAELRRRRLQKLESPVAH",
# TM129_HUMAN
"MDSPEVTFTLAYLVFAVCFVFTPNEFHAAGLTVQNLLSGWLGSEDAAFVPFHLRRTAATLLCHSLLPLGYYVGMCLAASEKRLHALSQAPEAWRLFLLLAVTLPSIACILIYYWSRDRWACHPLARTLALYALPQSGWQAVASSVNTEFRRIDKFATGAPGARVIVTDTWVMKVTTYRVHVAQQQDVHLTVTESRQHELSPDSNLPVQLLTIRVASTNPAVQAFDIWLNSTEYGELCEKLRAPIRRAAHVVIHQSLGDLFLETFASLVEVNPAYSVPSSQELEACIGCMQTRASVKLVKTCQEAATGECQQCYCRPMWCLTCMGKWFASRQDPLRPDTWLASRVPCPTCRARFCILDVCTVR",
# TOPRS_HUMAN
"MGSQPPLGSPLSREEGEAPPPAPASEGRRRSRRVRLRGSCRHRPSFLGCRELAASAPARPAPASSEIMASAAKEFKMDNFSPKAGTSKLQQTVPADASPDSKCPICLDRFDNVSYLDRCLHKFCFRCVQEWSKNKAECPLCKQPFDSIFHSVRAEDDFKEYVLRPSYNGSFVTPDRRFRYRTTLTRERNASVYSPSGPVNRRTTTPPDSGVLFEGLGISTRPRDVEIPQFMRQIAVRRPTTADERSLRKIQEQDIINFRRTLYRAGARVRNIEDGGRYRDISAEFFRRNPACLHRLVPWLKRELTVLFGAHGSLVNIVQHIIMSNVTRYDLESQAFVSDLRPFLLNRTEHFIHEFISFARSPFNMAAFDQHANYDCPAPSYEEGSHSDSSVITISPDEAETQELDINVATVSQAPWDDETPGPSYSSSEQVHVTMSSLLNTSDSSDEELVTGGATSQIQGVQTNDDLNNDSDDSSDNCVIVGFVKPLAERTPELVELSSDSEDLGSYEKMETVKTQEQEQSYSSGDSDVSRCSSPHSVLGKDEQINKGHCDSSTRIKSKKEEKRSTSLSSPRNLNSSVRGDRVYSPYNHRHRKRGRSRSSDSRSQSRSGHDQKNHRKHHGKKRMKSKRSRSRESSRPRGRRDKKRSRTRDSSWSRRSQTLSLSSESTSRSRSRSSDHGKRRSRSRNRDRYYLRNNYGSRYKWEYTYYSRNKDRDGYESSYRRRTLSRAHYSRQSSSPEFRVQSFSERTNARKKNNHSERKYYYYERHRSRSLSSNRSRTASTGTDRVRNEKPGGKRKYKTRHLEGTNEVAQPSREFASKAKDSHYQKSSSKLDGNYKNESDTFSDSRSSDRETKHKRRKRKTRSLSVEIVYEGKATDTTKHHKKKKKKHKKKHKKHHGDNASRSPVVITIDSDSDKDSEVKEDTECDNSGPQDPLQNEFLAPSLEPFETKDVVTIEAEFGVLDKECDIATLSNNLNNANKTVDNIPPLAASVEQTLDVREESTFVSDLENQPSNIVSLQTEPSRQLPSPRTSLMSVCLGRDCDMS",
# TRAF2_HUMAN
"MAAASVTPPGSLELLQPGFSKTLLGTKLEAKYLCSACRNVLRRPFQAQCGHRYCSFCLASILSSGPQNCAACVHEGIYEEGISILESSSAFPDNAARREVESLPAVCPSDGCTWKGTLKEYESCHEGRCPLMLTECPACKGLVRLGEKERHLEHECPERSLSCRHCRAPCCGADVKAHHEVCPKFPLTCDGCGKKKIPREKFQDHVKTCGKCRVPCRFHAIGCLETVEGEKQQEHEVQWLREHLAMLLSSVLEAKPLLGDQSHAGSELLQRCESLEKKTATFENIVCVLNREVERVAMTAEACSRQHRLDQDKIEALSSKVQQLERSIGLKDLAMADLEQKVLEMEASTYDGVFIWKISDFARKRQEAVAGRIPAIFSPAFYTSRYGYKMCLRIYLNGDGTGRGTHLSLFFVVMKGPNDALLRWPFNQKVTLMLLDQNNREHVIDAFRPDVTSSSFQRPVNDMNIASGCPLFCPVSKMEAKNSYVRDDAIFIKAIVDLTGL",
# TRAF3_HUMAN
"MESSKKMDSPGALQTNPPLKLHTDRSAGTPVFVPEQGGYKEKFVKTVEDKYKCEKCHLVLCSPKQTECGHRFCESCMAALLSSSSPKCTACQESIVKDKVFKDNCCKREILALQIYCRNESRGCAEQLMLGHLLVHLKNDCHFEELPCVRPDCKEKVLRKDLRDHVEKACKYREATCSHCKSQVPMIALQKHEDTDCPCVVVSCPHKCSVQTLLRSELSAHLSECVNAPSTCSFKRYGCVFQGTNQQIKAHEASSAVQHVNLLKEWSNSLEKKVSLLQNESVEKNKSIQSLHNQICSFEIEIERQKEMLRNNESKILHLQRVIDSQAEKLKELDKEIRPFRQNWEEADSMKSSVESLQNRVTELESVDKSAGQVARNTGLLESQLSRHDQMLSVHDIRLADMDLRFQVLETASYNGVLIWKIRDYKRRKQEAVMGKTLSLYSQPFYTGYFGYKMCARVYLNGDGMGKGTHLSLFFVIMRGEYDALLPWPFKQKVTLMLMDQGSSRRHLGDAFKPDPNSSSFKKPTGEMNIASGCPVFVAQTVLENGTYIKDDTIFIKVIVDTSDLPDP",
# TRAF4_HUMAN
"MPGFDYKFLEKPKRRLLCPLCGKPMREPVQVSTCGHRFCDTCLQEFLSEGVFKCPEDQLPLDYAKIYPDPELEVQVLGLPIRCIHSEEGCRWSGPLRHLQGHLNTCSFNVIPCPNRCPMKLSRRDLPAHLQHDCPKRRLKCEFCGCDFSGEAYESHEGMCPQESVYCENKCGARMMRRLLAQHATSECPKRTQPCTYCTKEFVFDTIQSHQYQCPRLPVACPNQCGVGTVAREDLPGHLKDSCNTALVLCPFKDSGCKHRCPKLAMARHVEESVKPHLAMMCALVSRQRQELQELRRELEELSVGSDGVLIWKIGSYGRRLQEAKAKPNLECFSPAFYTHKYGYKLQVSAFLNGNGSGEGTHLSLYIRVLPGAFDNLLEWPFARRVTFSLLDQSDPGLAKPQHVTETFHPDPNWKNFQKPGTWRGSLDESSLGFGYPKFISHQDIRKRNYVRDDAVFIRAAVELPRKILS",
# TRAF5_HUMAN
"MAYSEEHKGMPCGFIRQNSGNSISLDFEPSIEYQFVERLEERYKCAFCHSVLHNPHQTGCGHRFCQHCILSLRELNTVPICPVDKEVIKSQEVFKDNCCKREVLNLYVYCSNAPGCNAKVILGRYQDHLQQCLFQPVQCSNEKCREPVLRKDLKEHLSASCQFRKEKCLYCKKDVVVINLQNHEENLCPEYPVFCPNNCAKIILKTEVDEHLAVCPEAEQDCPFKHYGCAVTDKRRNLQQHEHSALREHMRLVLEKNVQLEEQISDLHKSLEQKESKIQQLAETIKKLEKEFKQFAQLFGKNGSFLPNIQVFASHIDKSAWLEAQVHQLLQMVNQQQNKFDLRPLMEAVDTVKQKITLLENNDQRLAVLEEETNKHDTHINIHKAQLSKNEERFKLLEGTCYNGKLIWKVTDYKMKKREAVDGHTVSIFSQSFYTSRCGYRLCARAYLNGDGSGRGSHLSLYFVVMRGEFDSLLQWPFRQRVTLMLLDQSGKKNIMETFKPDPNSSSFKRPDGEMNIASGCPRFVAHSVLENAKNAYIKDDTLFLKVAVDLTDLEDL",
# TRAF6_HUMAN
"MSLLNCENSCGSSQSESDCCVAMASSCSAVTKDDSVGGTASTGNLSSSFMEEIQGYDVEFDPPLESKYECPICLMALREAVQTPCGHRFCKACIIKSIRDAGHKCPVDNEILLENQLFPDNFAKREILSLMVKCPNEGCLHKMELRHLEDHQAHCEFALMDCPQCQRPFQKFHINIHILKDCPRRQVSCDNCAASMAFEDKEIHDQNCPLANVICEYCNTILIREQMPNHYDLDCPTAPIPCTFSTFGCHEKMQRNHLARHLQENTQSHMRMLAQAVHSLSVIPDSGYISEVRNFQETIHQLEGRLVRQDHQIRELTAKMETQSMYVSELKRTIRTLEDKVAEIEAQQCNGIYIWKIGNFGMHLKCQEEEKPVVIHSPGFYTGKPGYKLCMRLHLQLPTAQRCANYISLFVHTMQGEYDSHLPWPFQGTIRLTILDQSEAPVRQNHEEIMDAKPELLAFQRPTIPRNPKGFGYVTFMHLEALRQRTFIKDDTLLVRCEVSTRFDMGSLRREGFQPRSTDAGV",
# TRAF7_HUMAN
"MSSGKSARYNRFSGGPSNLPTPDVTTGTRMETTFGPAFSAVTTITKADGTSTYKQHCRTPSSSSTLAYSPRDEEDSMPPISTPRRSDSAISVRSLHSESSMSLRSTFSLPEEEEEPEPLVFAEQPSVKLCCQLCCSVFKDPVITTCGHTFCRRCALKSEKCPVDNVKLTVVVNNIAVAEQIGELFIHCRHGCRVAGSGKPPIFEVDPRGCPFTIKLSARKDHEGSCDYRPVRCPNNPSCPPLLRMNLEAHLKECEHIKCPHSKYGCTFIGNQDTYETHLETCRFEGLKEFLQQTDDRFHEMHVALAQKDQEIAFLRSMLGKLSEKIDQLEKSLELKFDVLDENQSKLSEDLMEFRRDASMLNDELSHINARLNMGILGSYDPQQIFKCKGTFVGHQGPVWCLCVYSMGDLLFSGSSDKTIKVWDTCTTYKCQKTLEGHDGIVLALCIQGCKLYSGSADCTIIVWDIQNLQKVNTIRAHDNPVCTLVSSHNVLFSGSLKAIKVWDIVGTELKLKKELTGLNHWVRALVAAQSYLYSGSYQTIKIWDIRTLDCIHVLQTSGGSVYSIAVTNHHIVCGTYENLIHVWDIESKEQVRTLTGHVGTVYALAVISTPDQTKVFSASYDRSLRVWSMDNMICTQTLLRHQGSVTALAVSRGRLFSGAVDSTVKVWTC",
# TRAIP_HUMAN
"MPIRALCTICSDFFDHSRDVAAIHCGHTFHLQCLIQWFETAPSRTCPQCRIQVGKRTIINKLFFDLAQEEENVLDAEFLKNELDNVRAQLSQKDKEKRDSQVIIDTLRDTLEERNATVVSLQQALGKAEMLCSTLKKQMKYLEQQQDETKQAQEEARRLRSKMKTMEQIELLLQSQRPEVEEMIRDMGVGQSAVEQLAVYCVSLKKEYENLKEARKASGEVADKLRKDLFSSRSKLQTVYSELDQAKLELKSAQKDLQSADKEIMSLKKKLTMLQETLNLPPVASETVDRLVLESPAPVEVNLKLRRPSFRDDIDLNATFDVDTPPARPSSSQHGYYEKLCLEKSHSPIQDVPKKICKGPRKESQLSLGGQSCAGEPDEELVGAFPIFVRNAILGQKQPKRPRSESSCSKDVVRTGFDGLGGRTKFIQPTDTVMIRPLPVKPKTKVKQRVRVKTVPSLFQAKLDTFLWS",
# TRI10_HUMAN
"MASAASVTSLADEVNCPICQGTLREPVTIDCGHNFCRACLTRYCEIPGPDLEESPTCPLCKEPFRPGSFRPNWQLANVVENIERLQLVSTLGLGEEDVCQEHGEKIYFFCEDDEMQLCVVCREAGEHATHTMRFLEDAAAPYREQIHKCLKCLRKEREEIQEIQSRENKRMQVLLTQVSTKRQQVISEFAHLRKFLEEQQSILLAQLESQDGDILRQRDEFDLLVAGEICRFSALIEELEEKNERPARELLTDIRSTLIRCETRKCRKPVAVSPELGQRIRDFPQQALPLQREMKMFLEKLCFELDYEPAHISLDPQTSHPKLLLSEDHQRAQFSYKWQNSPDNPQRFDRATCVLAHTGITGGRHTWVVSIDLAHGGSCTVGVVSEDVQRKGELRLRPEEGVWAVRLAWGFVSALGSFPTRLTLKEQPRQVRVSLDYEVGWVTFTNAVTREPIYTFTASFTRKVIPFFGLWGRGSSFSLSS",
# TRI11_HUMAN
"MAAPDLSTNLQEEATCAICLDYFTDPVMTDCGHNFCRECIRRCWGQPEGPYACPECRELSPQRNLRPNRPLAKMAEMARRLHPPSPVPQGVCPAHREPLAAFCGDELRLLCAACERSGEHWAHRVRPLQDAAEDLKAKLEKSLEHLRKQMQDALLFQAQADETCVLWQKMVESQRQNVLGEFERLRRLLAEEEQQLLQRLEEEELEVLPRLREGAAHLGQQSAHLAELIAELEGRCQLPALGLLQDIKDALRRVQDVKLQPPEVVPMELRTVCRVPGLVETLRRFRGDVTLDPDTANPELILSEDRRSVQRGDLRQALPDSPERFDPGPCVLGQERFTSGRHYWEVEVGDRTSWALGVCRENVNRKEKGELSAGNGFWILVFLGSYYNSSERALAPLRDPPRRVGIFLDYEAGHLSFYSATDGSLLFIFPEIPFSGTLRPLFSPLSSSPTPMTICRPKGGSGDTLAPQ",
# TRI13_HUMAN
"MELLEEDLTCPICCSLFDDPRVLPCSHNFCKKCLEGILEGSVRNSLWRPAPFKCPTCRKETSATGINSLQVNYSLKGIVEKYNKIKISPKMPVCKGHLGQPLNIFCLTDMQLICGICATRGEHTKHVFCSIEDAYAQERDAFESLFQSFETWRRGDALSRLDTLETSKRKSLQLLTKDSDKVKEFFEKLQHTLDQKKNEILSDFETMKLAVMQAYDPEINKLNTILQEQRMAFNIAEAFKDVSEPIVFLQQMQEFREKIKVIKETPLPPSNLPASPLMKNFDTSQWEDIKLVDVDKLSLPQDTGTFISKIPWSFYKLFLLILLLGLVIVFGPTMFLEWSLFDDLATWKGCLSNFSSYLTKTADFIEQSVFYWEQVTDGFFIFNERFKNFTLVVLNNVAEFVCKYKLL",
# TRI15_HUMAN
"MPATPSLKVVHELPACTLCAGPLEDAVTIPCGHTFCRLCLPALSQMGAQSSGKILLCPLCQEEEQAETPMAPVPLGPLGETYCEEHGEKIYFFCENDAEFLCVFCREGPTHQAHTVGFLDEAIQPYRDRLRSRLEALSTERDEIEDVKCQEDQKLQVLLTQIESKKHQVETAFERLQQELEQQRCLLLARLRELEQQIWKERDEYITKVSEEVTRLGAQVKELEEKCQQPASELLQDVRVNQSRCEMKTFVSPEAISPDLVKKIRDFHRKILTLPEMMRMFSENLAHHLEIDSGVITLDPQTASRSLVLSEDRKSVRYTRQKKSLPDSPLRFDGLPAVLGFPGFSSGRHRWQVDLQLGDGGGCTVGVAGEGVRRKGEMGLSAEDGVWAVIISHQQCWASTSPGTDLPLSEIPRGVRVALDYEAGQVTLHNAQTQEPIFTFTASFSGKVFPFFAVWKKGSCLTLKG",
# TRI17_HUMAN
"MEAVELARKLQEEATCSICLDYFTDPVMTTCGHNFCRACIQLSWEKARGKKGRRKRKGSFPCPECREMSPQRNLLPNRLLTKVAEMAQQHPGLQKQDLCQEHHEPLKLFCQKDQSPICVVCRESREHRLHRVLPAEEAVQGYKLKLEEDMEYLREQITRTGNLQAREEQSLAEWQGKVKERRERIVLEFEKMNLYLVEEEQRLLQALETEEEETASRLRESVACLDRQGHSLELLLLQLEERSTQGPLQMLQDMKEPLSRKNNVSVQCPEVAPPTRPRTVCRVPGQIEVLRGFLEDVVPDATSAYPYLLLYESRQRRYLGSSPEGSGFCSKDRFVAYPCAVGQTAFSSGRHYWEVGMNITGDALWALGVCRDNVSRKDRVPKCPENGFWVVQLSKGTKYLSTFSALTPVMLMEPPSHMGIFLDFEAGEVSFYSVSDGSHLHTYSQATFPGPLQPFFCLGAPKSGQMVISTVTMWVKG",
# TRIM2_HUMAN
"MASEGTNIPSPVVRQIDKQFLICSICLERYKNPKVLPCLHTFCERCLQNYIPAHSLTLSCPVCRQTSILPEKGVAALQNNFFITNLMDVLQRTPGSNAEESSILETVTAVAAGKPLSCPNHDGNVMEFYCQSCETAMCRECTEGEHAEHPTVPLKDVVEQHKASLQVQLDAVNKRLPEIDSALQFISEIIHQLTNQKASIVDDIHSTFDELQKTLNVRKSVLLMELEVNYGLKHKVLQSQLDTLLQGQESIKSCSNFTAQALNHGTETEVLLVKKQMSEKLNELADQDFPLHPRENDQLDFIVETEGLKKSIHNLGTILTTNAVASETVATGEGLRQTIIGQPMSVTITTKDKDGELCKTGNAYLTAELSTPDGSVADGEILDNKNGTYEFLYTVQKEGDFTLSLRLYDQHIRGSPFKLKVIRSADVSPTTEGVKRRVKSPGSGHVKQKAVKRPASMYSTGKRKENPIEDDLIFRVGTKGRNKGEFTNLQGVAASTNGKILIADSNNQCVQIFSNDGQFKSRFGIRGRSPGQLQRPTGVAVHPSGDIIIADYDNKWVSIFSSDGKFKTKIGSGKLMGPKGVSVDRNGHIIVVDNKACCVFIFQPNGKIVTRFGSRGNGDRQFAGPHFAAVNSNNEIIITDFHNHSVKVFNQEGEFMLKFGSNGEGNGQFNAPTGVAVDSNGNIIVADWGNSRIQVFDGSGSFLSYINTSADPLYGPQGLALTSDGHVVVADSGNHCFKVYRYLQ",
# RO52_HUMAN
"MASAARLTMMWEEVTCPICLDPFVEPVSIECGHSFCQECISQVGKGGGSVCPVCRQRFLLKNLRPNRQLANMVNNLKEISQEAREGTQGERCAVHGERLHLFCEKDGKALCWVCAQSRKHRDHAMVPLEEAAQEYQEKLQVALGELRRKQELAEKLEVEIAIKRADWKKTVETQKSRIHAEFVQQKNFLVEEEQRQLQELEKDEREQLRILGEKEAKLAQQSQALQELISELDRRCHSSALELLQEVIIVLERSESWNLKDLDITSPELRSVCHVPGLKKMLRTCAVHITLDPDTANPWLILSEDRRQVRLGDTQQSIPGNEERFDSYPMVLGAQHFHSGKHYWEVDVTGKEAWDLGVCRDSVRRKGHFLLSSKSGFWTIWLWNKQKYEAGTYPQTPLHLQVPPCQVGIFLDYEAGMVSFYNITDHGSLIYSFSECAFTGPLRPFFSPGFNDGGKNTAPLTLCPLNIGSQGSTDY",
# TRI22_HUMAN
"MDFSVKVDIEKEVTCPICLELLTEPLSLDCGHSFCQACITAKIKESVIISRGESSCPVCQTRFQPGNLRPNRHLANIVERVKEVKMSPQEGQKRDVCEHHGKKLQIFCKEDGKVICWVCELSQEHQGHQTFRINEVVKECQEKLQVALQRLIKEDQEAEKLEDDIRQERTAWKNYIQIERQKILKGFNEMRVILDNEEQRELQKLEEGEVNVLDNLAAATDQLVQQRQDASTLISDLQRRLRGSSVEMLQDVIDVMKRSESWTLKKPKSVSKKLKSVFRVPDLSGMLQVLKELTDVQYYWVDVMLNPGSATSNVAISVDQRQVKTVRTCTFKNSNPCDFSAFGVFGCQYFSSGKYYWEVDVSGKIAWILGVHSKISSLNKRKSSGFAFDPSVNYSKVYSRYRPQYGYWVIGLQNTCEYNAFEDSSSSDPKVLTLFMAVPPCRIGVFLDYEAGIVSFFNVTNHGALIYKFSGCRFSRPAYPYFNPWNCLVPMTVCPPSS",
# TRI23_HUMAN
"MATLVVNKLGAGVDSGRQGSRGTAVVKVLECGVCEDVFSLQGDKVPRLLLCGHTVCHDCLTRLPLHGRAIRCPFDRQVTDLGDSGVWGLKKNFALLELLERLQNGPIGQYGAAEESIGISGESIIRCDEDEAHLASVYCTVCATHLCSECSQVTHSTKTLAKHRRVPLADKPHEKTMCSQHQVHAIEFVCLEEGCQTSPLMCCVCKEYGKHQGHKHSVLEPEANQIRASILDMAHCIRTFTEEISDYSRKLVGIVQHIEGGEQIVEDGIGMAHTEHVPGTAENARSCIRAYFYDLHETLCRQEEMALSVVDAHVREKLIWLRQQQEDMTILLSEVSAACLHCEKTLQQDDCRVVLAKQEITRLLETLQKQQQQFTEVADHIQLDASIPVTFTKDNRVHIGPKMEIRVVTLGLDGAGKTTILFKLKQDEFMQPIPTIGFNVETVEYKNLKFTIWDVGGKHKLRPLWKHYYLNTQAVVFVVDSSHRDRISEAHSELAKLLTEKELRDALLLIFANKQDVAGALSVEEITELLSLHKLCCGRSWYIQGCDARSGMGLYEGLDWLSRQLVAAGVLDVA",
# TIF1A_HUMAN
"MEVAVEKAVAAAAAASAAASGGPSAAPSGENEAESRQGPDSERGGEAARLNLLDTCAVCHQNIQSRAPKLLPCLHSFCQRCLPAPQRYLMLPAPMLGSAETPPPVPAPGSPVSGSSPFATQVGVIRCPVCSQECAERHIIDNFFVKDTTEVPSSTVEKSNQVCTSCEDNAEANGFCVECVEWLCKTCIRAHQRVKFTKDHTVRQKEEVSPEAVGVTSQRPVFCPFHKKEQLKLYCETCDKLTCRDCQLLEHKEHRYQFIEEAFQNQKVIIDTLITKLMEKTKYIKFTGNQIQNRIIEVNQNQKQVEQDIKVAIFTLMVEINKKGKALLHQLESLAKDHRMKLMQQQQEVAGLSKQLEHVMHFSKWAVSSGSSTALLYSKRLITYRLRHLLRARCDASPVTNNTIQFHCDPSFWAQNIINLGSLVIEDKESQPQMPKQNPVVEQNSQPPSGLSSNQLSKFPTQISLAQLRLQHMQQQVMAQRQQVQRRPAPVGLPNPRMQGPIQQPSISHQQPPPRLINFQNHSPKPNGPVLPPHPQQLRYPPNQNIPRQAIKPNPLQMAFLAQQAIKQWQISSGQGTPSTTNSTSSTPSSPTITSAAGYDGKAFGSPMIDLSSPVGGSYNLPSLPDIDCSSTIMLDNIVRKDTNIDHGQPRPPSNRTVQSPNSSVPSPGLAGPVTMTSVHPPIRSPSASSVGSRGSSGSSSKPAGADSTHKVPVVMLEPIRIKQENSGPPENYDFPVVIVKQESDEESRPQNANYPRSILTSLLLNSSQSSTSEETVLRSDAPDSTGDQPGLHQDNSSNGKSEWLDPSQKSPLHVGETRKEDDPNEDWCAVCQNGGELLCCEKCPKVFHLSCHVPTLTNFPSGEWICTFCRDLSKPEVEYDCDAPSHNSEKKKTEGLVKLTPIDKRKCERLLLFLYCHEMSLAFQDPVPLTVPDYYKIIKNPMDLSTIKKRLQEDYSMYSKPEDFVADFRLIFQNCAEFNEPDSEVANAGIKLENYFEELLKNLYPEKRFPKPEFRNESEDNKFSDDSDDDFVQPRKKRLKSIEERQLLK",
# TRI25_HUMAN
"MAELCPLAEELSCSICLEPFKEPVTTPCGHNFCGSCLNETWAVQGSPYLCPQCRAVYQARPQLHKNTVLCNVVEQFLQADLAREPPADVWTPPARASAPSPNAQVACDHCLKEAAVKTCLVCMASFCQEHLQPHFDSPAFQDHPLQPPVRDLLRRKCSQHNRLREFFCPEHSECICHICLVEHKTCSPASLSQASADLEATLRHKLTVMYSQINGASRALDDVRNRQQDVRMTANRKVEQLQQEYTEMKALLDASETTSTRKIKEEEKRVNSKFDTIYQILLKKKSEIQTLKEEIEQSLTKRDEFEFLEKASKLRGISTKPVYIPEVELNHKLIKGIHQSTIDLKNELKQCIGRLQEPTPSSGDPGEHDPASTHKSTRPVKKVSKEEKKSKKPPPVPALPSKLPTFGAPEQLVDLKQAGLEAAAKATSSHPNSTSLKAKVLETFLAKSRPELLEYYIKVILDYNTAHNKVALSECYTVASVAEMPQNYRPHPQRFTYCSQVLGLHCYKKGIHYWEVELQKNNFCGVGICYGSMNRQGPESRLGRNSASWCVEWFNTKISAWHNNVEKTLPSTKATRVGVLLNCDHGFVIFFAVADKVHLMYKFRVDFTEALYPAFWVFSAGATLSICSPK",
# TRI26_HUMAN
"MATSAPLRSLEEEVTCSICLDYLRDPVTIDCGHVFCRSCTTDVRPISGSRPVCPLCKKPFKKENIRPVWQLASLVENIERLKVDKGRQPGEVTREQQDAKLCERHREKLHYYCEDDGKLLCVMCRESREHRPHTAVLMEKAAQPHREKILNHLSTLRRDRDKIQGFQAKGEADILAALKKLQDQRQYIVAEFEQGHQFLREREEHLLEQLAKLEQELTEGREKFKSRGVGELARLALVISELEGKAQQPAAELMQDTRDFLNRYPRKKFWVGKPIARVVKKKTGEFSDKLLSLQRGLREFQGKLLRDLEYKTVSVTLDPQSASGYLQLSEDWKCVTYTSLYKSAYLHPQQFDCEPGVLGSKGFTWGKVYWEVEVEREGWSEDEEEGDEEEEGEEEEEEEEAGYGDGYDDWETDEDEESLGDEEEEEEEEEEEVLESCMVGVARDSVKRKGDLSLRPEDGVWALRLSSSGIWANTSPEAELFPALRPRRVGIALDYEGGTVTFTNAESQELIYTFTATFTRRLVPFLWLKWPGTRLLLRP",
# TRI27_HUMAN
"MASGSVAECLQQETTCPVCLQYFAEPMMLDCGHNICCACLARCWGTAETNVSCPQCRETFPQRHMRPNRHLANVTQLVKQLRTERPSGPGGEMGVCEKHREPLKLYCEEDQMPICVVCDRSREHRGHSVLPLEEAVEGFKEQIQNQLDHLKRVKDLKKRRRAQGEQARAELLSLTQMEREKIVWEFEQLYHSLKEHEYRLLARLEELDLAIYNSINGAITQFSCNISHLSSLIAQLEEKQQQPTRELLQDIGDTLSRAERIRIPEPWITPPDLQEKIHIFAQKCLFLTESLKQFTEKMQSDMEKIQELREAQLYSVDVTLDPDTAYPSLILSDNLRQVRYSYLQQDLPDNPERFNLFPCVLGSPCFIAGRHYWEVEVGDKAKWTIGVCEDSVCRKGGVTSAPQNGFWAVSLWYGKEYWALTSPMTALPLRTPLQRVGIFLDYDAGEVSFYNVTERCHTFTFSHATFCGPVRPYFSLSYSGGKSAAPLIICPMSGIDGFSGHVGNHGHSMETSP",
# TRIM3_HUMAN
"MAKREDSPGPEVQPMDKQFLVCSICLDRYQCPKVLPCLHTFCERCLQNYIPAQSLTLSCPVCRQTSILPEQGVSALQNNFFISSLMEAMQQAPDGAHDPEDPHPLSVVAGRPLSCPNHEGKTMEFYCEACETAMCGECRAGEHREHGTVLLRDVVEQHKAALQRQLEAVRGRLPQLSAAIALVGGISQQLQERKAEALAQISAAFEDLEQALQQRKQALVSDLETICGAKQKVLQSQLDTLRQGQEHIGSSCSFAEQALRLGSAPEVLLVRKHMRERLAALAAQAFPERPHENAQLELVLEVDGLRRSVLNLGALLTTSATAHETVATGEGLRQALVGQPASLTVTTKDKDGRLVRTGSAELRAEITGPDGTRLPVPVVDHKNGTYELVYTARTEGELLLSVLLYGQPVRGSPFRVRALRPGDLPPSPDDVKRRVKSPGGPGSHVRQKAVRRPSSMYSTGGKRKDNPIEDELVFRVGSRGREKGEFTNLQGVSAASSGRIVVADSNNQCIQVFSNEGQFKFRFGVRGRSPGQLQRPTGVAVDTNGDIIVADYDNRWVSIFSPEGKFKTKIGAGRLMGPKGVAVDRNGHIIVVDNKSCCVFTFQPNGKLVGRFGGRGATDRHFAGPHFVAVNNKNEIVVTDFHNHSVKVYSADGEFLFKFGSHGEGNGQFNAPTGVAVDSNGNIIVADWGNSRIQVFDSSGSFLSYINTSAEPLYGPQGLALTSDGHVVVADAGNHCFKAYRYLQ",
# TRI31_HUMAN
"MASGQFVNKLQEEVICPICLDILQKPVTIDCGHNFCLKCITQIGETSCGFFKCPLCKTSVRKNAIRFNSLLRNLVEKIQALQASEVQSKRKEATCPRHQEMFHYFCEDDGKFLCFVCRESKDHKSHNVSLIEEAAQNYQGQIQEQIQVLQQKEKETVQVKAQGVHRVDVFTDQVEHEKQRILTEFELLHQVLEEEKNFLLSRIYWLGHEGTEAGKHYVASTEPQLNDLKKLVDSLKTKQNMPPRQLLEDIKVVLCRSEEFQFLNPTPVPLELEKKLSEAKSRHDSITGSLKKFKDQLQADRKKDENRFFKSMNKNDMKSWGLLQKNNHKMNKTSEPGSSSAGGRTTSGPPNHHSSAPSHSLFRASSAGKVTFPVCLLASYDEISGQGASSQDTKTFDVALSEELHAALSEWLTAIRAWFCEVPSS",
# TRI32_HUMAN
"MAAAAASHLNLDALREVLECPICMESFTEEQLRPKLLHCGHTICRQCLEKLLASSINGVRCPFCSKITRITSLTQLTDNLTVLKIIDTAGLSEAVGLLMCRSCGRRLPRQFCRSCGLVLCEPCREADHQPPGHCTLPVKEAAEERRRDFGEKLTRLRELMGELQRRKAALEGVSKDLQARYKAVLQEYGHEERRVQDELARSRKFFTGSLAEVEKSNSQVVEEQSYLLNIAEVQAVSRCDYFLAKIKQADVALLEETADEEEPELTASLPRELTLQDVELLKVGHVGPLQIGQAVKKPRTVNVEDSWAMEATASAASTSVTFREMDMSPEEVVASPRASPAKQRGPEAASNIQQCLFLKKMGAKGSTPGMFNLPVSLYVTSQGEVLVADRGNYRIQVFTRKGFLKEIRRSPSGIDSFVLSFLGADLPNLTPLSVAMNCQGLIGVTDSYDNSLKVYTLDGHCVACHRSQLSKPWGITALPSGQFVVTDVEGGKLWCFTVDRGSGVVKYSCLCSAVRPKFVTCDAEGTVYFTQGLGLNLENRQNEHHLEGGFSIGSVGPDGQLGRQISHFFSENEDFRCIAGMCVDARGDLIVADSSRKEILHFPKGGGYSVLIREGLTCPVGIALTPKGQLLVLDCWDHCIKIYSYHLRRYSTP",
# TRI33_HUMAN
"MAENKGGGEAESGGGGSGSAPVTAGAAGPAAQEAEPPLTAVLVEEEEEEGGRAGAEGGAAGPDDGGVAAASSGSAQAASSPAASVGTGVAGGAVSTPAPAPASAPAPGPSAGPPPGPPASLLDTCAVCQQSLQSRREAEPKLLPCLHSFCLRCLPEPERQLSVPIPGGSNGDIQQVGVIRCPVCRQECRQIDLVDNYFVKDTSEAPSSSDEKSEQVCTSCEDNASAVGFCVECGEWLCKTCIEAHQRVKFTKDHLIRKKEDVSESVGASGQRPVFCPVHKQEQLKLFCETCDRLTCRDCQLLEHKEHRYQFLEEAFQNQKGAIENLLAKLLEKKNYVHFAATQVQNRIKEVNETNKRVEQEIKVAIFTLINEINKKGKSLLQQLENVTKERQMKLLQQQNDITGLSRQVKHVMNFTNWAIASGSSTALLYSKRLITFQLRHILKARCDPVPAANGAIRFHCDPTFWAKNVVNLGNLVIESKPAPGYTPNVVVGQVPPGTNHISKTPGQINLAQLRLQHMQQQVYAQKHQQLQQMRMQQPPAPVPTTTTTTQQHPRQAAPQMLQQQPPRLISVQTMQRGNMNCGAFQAHQMRLAQNAARIPGIPRHSGPQYSMMQPHLQRQHSNPGHAGPFPVVSVHNTTINPTSPTTATMANANRGPTSPSVTAIELIPSVTNPENLPSLPDIPPIQLEDAGSSSLDNLLSRYISGSHLPPQPTSTMNPSPGPSALSPGSSGLSNSHTPVRPPSTSSTGSRGSCGSSGRTAEKTSLSFKSDQVKVKQEPGTEDEICSFSGGVKQEKTEDGRRSACMLSSPESSLTPPLSTNLHLESELDALASLENHVKIEPADMNESCKQSGLSSLVNGKSPIRSLMHRSARIGGDGNNKDDDPNEDWCAVCQNGGDLLCCEKCPKVFHLTCHVPTLLSFPSGDWICTFCRDIGKPEVEYDCDNLQHSKKGKTAQGLSPVDQRKCERLLLYLYCHELSIEFQEPVPASIPNYYKIIKKPMDLSTVKKKLQKKHSQHYQIPDDFVADVRLIFKNCERFNEMMKVVQVYADTQEINLKADSEVAQAGKAVALYFEDKLTEIYSDRTFAPLPEFEQEEDDGEVTEDSDEDFIQPRRKRLKSDERPVHIK",
# TRI34_HUMAN
"MASKILLNVQEEVTCPICLELLTEPLSLDCGHSLCRACITVSNKEAVTSMGGKSSCPVCGISYSFEHLQANQHLANIVERLKEVKLSPDNGKKRDLCDHHGEKLLLFCKEDRKVICWLCERSQEHRGHHTVLTEEVFKECQEKLQAVLKRLKKEEEEAEKLEADIREEKTSWKYQVQTERQRIQTEFDQLRSILNNEEQRELQRLEEEEKKTLDKFAEAEDELVQQKQLVRELISDVECRSQWSTMELLQDMSGIMKWSEIWRLKKPKMVSKKLKTVFHAPDLSRMLQMFRELTAVRCYWVDVTLNSVNLNLNLVLSEDQRQVISVPIWPFQCYNYGVLGSQYFSSGKHYWEVDVSKKTAWILGVYCRTYSRHMKYVVRRCANRQNLYTKYRPLFGYWVIGLQNKCKYGVFEESLSSDPEVLTLSMAVPPCRVGVFLDYEAGIVSFFNVTSHGSLIYKFSKCCFSQPVYPYFNPWNCPAPMTLCPPSS",
# TRI35_HUMAN
"MERSPDVSPGPSRSFKEELLCAVCYDPFRDAVTLRCGHNFCRGCVSRCWEVQVSPTCPVCKDRASPADLRTNHTLNNLVEKLLREEAEGARWTSYRFSRVCRLHRGQLSLFCLEDKELLCCSCQADPRHQGHRVQPVKDTAHDFRAKCRNMEHALREKAKAFWAMRRSYEAIAKHNQVEAAWLEGRIRQEFDKLREFLRVEEQAILDAMAEETRQKQLLADEKMKQLTEETEVLAHEIERLQMEMKEDDVSFLMKHKSRKRRLFCTMEPEPVQPGMLIDVCKYLGSLQYRVWKKMLASVESVPFSFDPNTAAGWLSVSDDLTSVTNHGYRVQVENPERFSSAPCLLGSRVFSQGSHAWEVALGGLQSWRVGVVRVRQDSGAEGHSHSCYHDTRSGFWYVCRTQGVEGDHCVTSDPATSPLVLAIPRRLRVELECEEGELSFYDAERHCHLYTFHARFGEVRPYFYLGGARGAGPPEPLRICPLHISVKEELDG",
# TRI36_HUMAN
"MSESGEMSEFGYIMELIAKGKVTIKNIERELICPACKELFTHPLILPCQHSICHKCVKELLLTLDDSFNDVGSDNSNQSSPRLRLPSPSMDKIDRINRPGWKRNSLTPRTTVFPCPGCEHDVDLGERGINGLFRNFTLETIVERYRQAARAATAIMCDLCKPPPQESTKSCMDCSASYCNECFKIHHPWGTIKAQHEYVGPTTNFRPKILMCPEHETERINMYCELCRRPVCHLCKLGGNHANHRVTTMSSAYKTLKEKLSKDIDYLIGKESQVKSQISELNLLMKETECNGERAKEEAITHFEKLFEVLEERKSSVLKAIDSSKKLRLDKFQTQMEEYQGLLENNGLVGYAQEVLKETDQSCFVQTAKQLHLRIQKATESLKSFRPAAQTSFEDYVVNTSKQTELLGELSFFSSGIDVPEINEEQSKVYNNALINWHHPEKDKADSYVLEYRKINRDDEMSWNEIEVCGTSKIIQDLENSSTYAFRVRAYKGSICSPCSRELILHTPPAPVFSFLFDEKCGYNNEHLLLNLKRDRVESRAGFNLLLAAERIQVGYYTSLDYIIGDTGITKGKHFWAFRVEPYSYLVKVGVASSDKLQEWLRSPRDAVSPRYEQDSGHDSGSEDACFDSSQPFTLVTIGMQKFFIPKSPTSSNEPENRVLPMPTSIGIFLDCDKGKVDFYDMDQMKCLYERQVDCSHTLYPAFALMGSGGIQLEEPITAKYLEYQEDM",
# TRI37_HUMAN
"MDEQSVESIAEVFRCFICMEKLRDARLCPHCSKLCCFSCIRRWLTEQRAQCPHCRAPLQLRELVNCRWAEEVTQQLDTLQLCSLTKHEENEKDKCENHHEKLSVFCWTCKKCICHQCALWGGMHGGHTFKPLAEIYEQHVTKVNEEVAKLRRRLMELISLVQEVERNVEAVRNAKDERVREIRNAVEMMIARLDTQLKNKLITLMGQKTSLTQETELLESLLQEVEHQLRSCSKSELISKSSEILMMFQQVHRKPMASFVTTPVPPDFTSELVPSYDSATFVLENFSTLRQRADPVYSPPLQVSGLCWRLKVYPDGNGVVRGYYLSVFLELSAGLPETSKYEYRVEMVHQSCNDPTKNIIREFASDFEVGECWGYNRFFRLDLLANEGYLNPQNDTVILRFQVRSPTFFQKSRDQHWYITQLEAAQTSYIQQINNLKERLTIELSRTQKSRDLSPPDNHLSPQNDDALETRAKKSACSDMLLEGGPTTASVREAKEDEEDEEKIQNEDYHHELSDGDLDLDLVYEDEVNQLDGSSSSASSTATSNTEENDIDEETMSGENDVEYNNMELEEGELMEDAAAAGPAGSSHGYVGSSSRISRRTHLCSAATSSLLDIDPLILIHLLDLKDRSSIENLWGLQPRPPASLLQPTASYSRKDKDQRKQQAMWRVPSDLKMLKRLKTQMAEVRCMKTDVKNTLSEIKSSSAASGDMQTSLFSADQAALAACGTENSGRLQDLGMELLAKSSVANCYIRNSTNKKSNSPKPARSSVAGSLSLRRAVDPGENSRSKGDCQTLSEGSPGSSQSGSRHSSPRALIHGSIGDILPKTEDRQCKALDSDAVVVAVFSGLPAVEKRRKMVTLGANAKGGHLEGLQMTDLENNSETGELQPVLPEGASAAPEEGMSSDSDIECDTENEEQEEHTSVGGFHDSFMVMTQPPDEDTHSSFPDGEQIGPEDLSFNTDENSGR",
# TRI38_HUMAN
"MASTTSTKKMMEEATCSICLSLMTNPVSINCGHSYCHLCITDFFKNPSQKQLRQETFCCPQCRAPFHMDSLRPNKQLGSLIEALKETDQEMSCEEHGEQFHLFCEDEGQLICWRCERAPQHKGHTTALVEDVCQGYKEKLQKAVTKLKQLEDRCTEQKLSTAMRITKWKEKVQIQRQKIRSDFKNLQCFLHEEEKSYLWRLEKEEQQTLSRLRDYEAGLGLKSNELKSHILELEEKCQGSAQKLLQNVNDTLSRSWAVKLETSEAVSLELHTMCNVSKLYFDVKKMLRSHQVSVTLDPDTAHHELILSEDRRQVTRGYTQENQDTSSRRFTAFPCVLGCEGFTSGRRYFEVDVGEGTGWDLGVCMENVQRGTGMKQEPQSGFWTLRLCKKKGYVALTSPPTSLHLHEQPLLVGIFLDYEAGVVSFYNGNTGCHIFTFPKASFSDTLRPYFQVYQYSPLFLPPPGD",
# TRI39_HUMAN
"MAETSLLEAGASAASTAAALENLQVEASCSVCLEYLKEPVIIECGHNFCKACITRWWEDLERDFPCPVCRKTSRYRSLRPNRQLGSMVEIAKQLQAVKRKIRDESLCPQHHEALSLFCYEDQEAVCLICAISHTHRAHTVVPLDDATQEYKEKLQKCLEPLEQKLQEITRCKSSEEKKPGELKRLVESRRQQILREFEELHRRLDEEQQVLLSRLEEEEQDILQRLRENAAHLGDKRRDLAHLAAEVEGKCLQSGFEMLKDVKSTLEKNIPRKFGGSLSTICPRDHKALLGLVKEINRCEKVKTMEVTSVSIELEKNFSNFPRQYFALRKILKQLIADVTLDPETAHPNLVLSEDRKSVKFVETRLRDLPDTPRRFTFYPCVLATEGFTSGRHYWEVEVGDKTHWAVGVCRDSVSRKGELTPLPETGYWRVRLWNGDKYAATTTPFTPLHIKVKPKRVGIFLDYEAGTLSFYNVTDRSHIYTFTDTFTEKLWPLFYPGIRAGRKNAAPLTIRPPTDWE",
# TRIM4_HUMAN
"MEAEDIQEELTCPICLDYFQDPVSIECGHNFCRGCLHRNWAPGGGPFPCPECRHPSAPAALRPNWALARLTEKTQRRRLGPVPPGLCGRHWEPLRLFCEDDQRPVCLVCRESQEHQTHAMAPIDEAFESYRTGNFDIHVDEWKRRLIRLLLYHFKQEEKLLKSQRNLVAKMKKVMHLQDVEVKNATQWKDKIKSQRMRISTEFSKLHNFLVEEEDLFLQRLNKEEEETKKKLNENTLKLNQTIASLKKLILEVGEKSQAPTLELLQNPKEVLTRSEIQDVNYSLEAVKVKTVCQIPLMKEMLKRFQVAVNLAEDTAHPKLVFSQEGRYVKNTASASSWPVFSSAWNYFAGWRNPQKTAFVERFQHLPCVLGKNVFTSGKHYWEVESRDSLEVAVGVCREDVMGITDRSKMSPDVGIWAIYWSAAGYWPLIGFPGTPTQQEPALHRVGVYLDRGTGNVSFYSAVDGVHLHTFSCSSVSRLRPFFWLSPLASLVIPPVTDRK",
# TRI40_HUMAN
"MIPLQKDNQEEGVCPICQESLKEAVSTNCGHLFCRVCLTQHVEKASASGVFCCPLCRKPCSEEVLGTGYICPNHQKRVCRFCEESRLLLCVECLVSPEHMSHHELTIENALSHYKERLNRRSRKLRKDIAELQRLKAQQEKKLQALQFQVDHGNHRLEAGPESQHQTREQLGALPQQWLGQLEHMPAEAARILDISRAVTQLRSLVIDLERTAKELDTNTLKNAGDLLNRSAPQKLEVIYPQLEKGVSELLLQPPQKL",
# TRI41_HUMAN
"MAAVAMTPNPVQTLQEEAVCAICLDYFTDPVSIGCGHNFCRVCVTQLWGGEDEEDRDELDREEEEEDGEEEEVEAVGAGAGWDTPMRDEDYEGDMEEEVEEEEEGVFWTSGMSRSSWDNMDYVWEEEDEEEDLDYYLGDMEEEDLRGEDEEDEEEVLEEVEEEDLDPVTPLPPPPAPRRCFTCPQCRKSFPRRSFRPNLQLANMVQVIRQMHPTPGRGSRVTDQGICPKHQEALKLFCEVDEEAICVVCRESRSHKQHSVVPLEEVVQEYKAKLQGHVEPLRKHLEAVQKMKAKEERRVTELKSQMKSELAAVASEFGRLTRFLAEEQAGLERRLREMHEAQLGRAGAAASRLAEQAAQLSRLLAEAQERSQQGGLRLLQDIKETFNRCEEVQLQPPEVWSPDPCQPHSHDFLTDAIVRKMSRMFCQAARVDLTLDPDTAHPALMLSPDRRGVRLAERRQEVADHPKRFSADCCVLGAQGFRSGRHYWEVEVGGRRGWAVGAARESTHHKEKVGPGGSSVGSGDASSSRHHHRRRRLHLPQQPLLQREVWCVGTNGKRYQAQSSTEQTLLSPSEKPRRFGVYLDYEAGRLGFYNAETLAHVHTFSAAFLGERVFPFFRVLSKGTRIKLCP",
# TRI42_HUMAN
"METAMCVCCPCCTWQRCCPQLCSCLCCKFIFTSERNCTCFPCPYKDERNCQFCHCTCSESPNCHWCCCSWANDPNCKCCCTASSNLNCYYYESRCCRNTIITFHKGRLRSIHTSSKTALRTGSSDTQVDEVKSIPANSHLVNHLNCPMCSRLRLHSFMLPCNHSLCEKCLRQLQKHAEVTENFFILICPVCDRSHCMPYSNKMQLPENYLHGRLTKRYMQEHGYLKWRFDRSSGPILCQVCRNKRIAYKRCITCRLNLCNDCLKAFHSDVAMQDHVFVDTSAEEQDEKICIHHPSSRIIEYCRNDNKLLCTFCKFSFHNGHDTISLIDACSERAASLFSAIAKFKAVRYEIDNDLMEFNILKNSFKADKEAKRKEIRNGFLKLRSILQEKEKIIMEQIENLEVSRQKEIEKYVYVTTMKVNEMDGLIAYSKEALKETGQVAFLQSAKILVDQIEDGIQTTYRPDPQLRLHSINYVPLDFVELSSAIHELFPTGPKKVRSSGDSLPSPYPVHSETMIARKVTFSTHSLGNQHIYQRSSSMLSFSNTDKKAKVGLEACGRAQSATPAKPTDGLYTYWSAGADSQSVQNSSSFHNWYSFNDGSVKTPGPIVIYQTLVYPRAAKVYWTCPAEDVDSFEMEFYEVITSPPNNVQMELCGQIRDIMQQNLELHNLTPNTEYVFKVRAINDNGPGQWSDICKVVTPDGHGKNRAKWGLLKNIQSALQKHF",
# TRI43_HUMAN
"MDSDFSHAFQKELTCVICLNYLVDPVTICCGHSFCRPCLCLSWEEAQSPANCPACREPSPKMDFKTNILLKNLVTIARKASLWQFLSSEKQICGTHRQTKKMFCDMDKSLLCLLCSNSQEHGAHKHHPIEEAAEEHREKLLKQMRILWKKIQENQRNLYEEGRTAFLWRGNVVLRAQMIRNEYRKLHPVLHKEEKQHLERLNKEYQEIFQQLQRSWVKMDQKSKHLKEMYQELMEMCHKPDVELLQDLGDIVARSESVLLHMPQPVNPELTAGPITGLVYRLNRFRVEISFHFEVTNHNIRLFEDVRSWMFRRGPLNSDRSDYFAAWGARVFSFGKHYWELDVDNSCDWALGVCNNSWIRKNSTMVNSEDIFLLLCLKVDNHFNLLTTSPVFPHYIEKPLGRVGVFLDFESGSVSFLNVTKSSLIWSYPAGSLTFPVRPFFYTGHR",
# TR43B_HUMAN
"MDSDFSHAFQKELTCVICLNYLVDPVTICCGHSFCRPCLCLSWEEAQSPANCPACREPSPKMDFKTNILLKNLVTIARKASLWQFLSSEKQICGTHRQTKKMFCDMDKSLLCLLCSNSQEHGAHKHYPIEEAAEEDREKLLKQMRILWKKIQENQRNLYEERRTAFLLRGDVVLRAQMIRNEYRKLHPVLHKEEKQHLERLNKEYQEIFQQLQRSWVKMDQKSKHLKEMYQELMEMCHKPEVELLQDLGDIVARSESVLLHMPQPVNPELTAGPITGLVYRLNRFRVEISFHFEVTNHNIRLFEDVRSWMFRRGPLNSDRSDYFAAWGARVFSFGKHYWELDVDNSCDWALGVCNNSWIRKNSTMVNSEDIFLLLCLKVDNHFNLLTTSPVFPHYIEKPLGRVGVFLDFESGSVSFLNVTKSSLIWSYPAGSLTFPVRPFFYTGHR",
# TRI45_HUMAN
"MSENRKPLLGFVSKLTSGTALGNSGKTHCPLCLGLFKAPRLLPCLHTVCTTCLEQLEPFSVVDIRGGDSDTSSEGSIFQELKPRSLQSQIGILCPVCDAQVDLPMGGVKALTIDHLAVNDVMLESLRGEGQGLVCDLCNDREVEKRCQTCKANLCHFCCQAHRRQKKTTYHTMVDLKDLKGYSRIGKPILCPVHPAEELRLFCEFCDRPVCQDCVVGEHREHPCDFTSNVIHKHGDSVWELLKGTQPHVEALEEALAQIHIINSALQKRVEAVAADVRTFSEGYIKAIEEHRDKLLKQLEDIRAQKENSLQLQKAQLEQLLADMRTGVEFTEHLLTSGSDLEILITKRVVVERLRKLNKVQYSTRPGVNDKIRFCPQEKAGQCRGYEIYGTINTKEVDPAKCVLQGEDLHRAREKQTASFTLLCKDAAGEIMGRGGDNVQVAVVPKDKKDSPVRTMVQDNKDGTYYISYTPKEPGVYTVWVCIKEQHVQGSPFTVMVRRKHRPHSGVFHCCTFCSSGGQKTARCACGGTMPGGYLGCGHGHKGHPGHPHWSCCGKFNEKSECTWTGGQSAPRSLLRTVAL",
# TRI46_HUMAN
"MAEGEDMQTFTSIMDALVRISTSMKNMEKELLCPVCQEMYKQPLVLPCTHNVCQACAREVLGQQGYIGHGGDPSSEPTSPASTPSTRSPRLSRRTLPKPDRLDRLLKSGFGTYPGRKRGALHPQVIMFPCPACQGDVELGERGLAGLFRNLTLERVVERYRQSVSVGGAILCQLCKPPPLEATKGCTECRATFCNECFKLFHPWGTQKAQHEPTLPTLSFRPKGLMCPDHKEEVTHYCKTCQRLVCQLCRVRRTHSGHKITPVLSAYQALKDKLTKSLTYILGNQDTVQTQICELEEAVRHTEVSGQQAKEEVSQLVRGLGAVLEEKRASLLQAIEECQQERLARLSAQIQEHRSLLDGSGLVGYAQEVLKETDQPCFVQAAKQLHNRIARATEALQTFRPAASSSFRHCQLDVGREMKLLTELNFLRVPEAPVIDTQRTFAYDQIFLCWRLPPHSPPAWHYTVEFRRTDVPAQPGPTRWQRREEVRGTSALLENPDTGSVYVLRVRGCNKAGYGEYSEDVHLHTPPAPVLHFFLDSRWGASRERLAISKDQRAVRSVPGLPLLLAADRLLTGCHLSVDVVLGDVAVTQGRSYWACAVDPASYLVKVGVGLESKLQESFQGAPDVISPRYDPDSGHDSGAEDATVEASPPFAFLTIGMGKILLGSGASSNAGLTGRDGPTAGCTVPLPPRLGICLDYERGRVSFLDAVSFRGLLECPLDCSGPVCPAFCFIGGGAVQLQEPVGTKPERKVTIGGFAKLD",
# TRI47_HUMAN
"MDGSGPFSCPICLEPLREPVTLPCGHNFCLACLGALWPHRGASGAGGPGGAARCPLCQEPFPDGLQLRKNHTLSELLQLRQGSGPGSGPGPAPALAPEPSAPSALPSVPEPSAPCAPEPWPAGEEPVRCDACPEGAALPAALSCLSCLASFCPAHLGPHERSPALRGHRLVPPLRRLEESLCPRHLRPLERYCRAERVCLCEACAAQEHRGHELVPLEQERALQEAEQSKVLSAVEDRMDELGAGIAQSRRTVALIKSAAVAERERVSRLFADAAAALQGFQTQVLGFIEEGEAAMLGRSQGDLRRQEEQRSRLSRARQNLSQVPEADSVSFLQELLALRLALEDGCGPGPGPPRELSFTKSSQAVRAVRDMLAVACVNQWEQLRGPGGNEDGPQKLDSEADAEPQDLESTNLLESEAPRDYFLKFAYIVDLDSDTADKFLQLFGTKGVKRVLCPINYPLSPTRFTHCEQVLGEGALDRGTYYWEVEIIEGWVSMGVMAEDFSPQEPYDRGRLGRNAHSCCLQWNGRSFSVWFHGLEAPLPHPFSPTVGVCLEYADRALAFYAVRDGKMSLLRRLKASRPRRGGIPASPIDPFQSRLDSHFAGLFTHRLKPAFFLESVDAHLQIGPLKKSCISVLKRR",
# TRI48_HUMAN
"MSRRIIVGTLQRTQRNMNSGISQVFQRELTCPICMNYFIDPVTIDCGHSFCRPCFYLNWQDIPILTQCFECIKTIQQRNLKTNIRLKKMASLARKASLWLFLSSEEQMCGIHRETKKMFCEVDRSLLCLLCSSSQEHRYHRHCPAEWAAEEHWEKLLKKMQSLWEKACENQRNLNVETTRISHWKAFGDILYRSESVLLHMPQPLNLALRAGPITGLRDRLNQF",
# TRI49_HUMAN
"MNSGILQVFQGELICPLCMNYFIDPVTIDCGHSFCRPCFYLNWQDIPFLVQCSECTKSTEQINLKTNIHLKKMASLARKVSLWLFLSSEEQMCGTHRETKKIFCEVDRSLLCLLCSSSQEHRYHRHRPIEWAAEEHREKLLQKMQSLWEKACENHRNLNVETTRTRCWKDYVNLRLEAIRAEYQKMPAFHHEEEKHNLEMLKKKGKEIFHRLHLSKAKMAHRMEILRGMYEELNEMCHKPDVELLQAFGDILHRSESVLLHMPQPLNPELSAGPITGLRDRLNQFRVHITLHHEEANNDIFLYEILRSMCIGCDHQDVPYFTATPRSFLAWGVQTFTSGKYYWEVHVGDSWNWAFGVCNMYRKEKNQNEKIDGKAGLFLLGCVKNDIQCSLFTTSPLMLQYIPKPTSRVGLFLDCEAKTVSFVDVNQSSLIYTIPNCSFSPPLRPIFCCIHF",
# TR49B_HUMAN
"MNSGILQVFQRELICPICMNYFIDPVTIDCGHSFCRPCFYLNWKDSPFLVQCSECTKSTGQINLKTNIHFKKMASLARKVSLWLFLSSEEQMCGTHRETKKMFCEVDRSLLCLLCSSSQEHRDHRHCPIESAAEEHQEKLLQKMQSLWEKACENHRNLNVETTRTRCWKDYVNLRLEAIRAEYQKMPAFHHEEEKHNLEMLKKKGKDIFHRLHLSKAKMAHRREILRGMYEELNEMCHKPDVELLQAFGDILHRSESVLLHMPQPLNPELSAGPITGLRDRLNQFRVHITLHHEEANSDIFLCEILRSMCIGCDHQDVPYFTATPRSFLAWGAQTFTSGKYYWEVHVGDSWNWAFGVCNMYWKEKNQNEKIDGEDGLFLLGCVKNDIQRSLFTTSPLLLQYIPRPTSRVGLFLDCEAKTVSFVDVNQSSLIYTIPNCSFSPPLRPIFCCIHF",
# TR49C_HUMAN
"MNSGILQVFQGELICPLCMNYFIDPVTIDCGHSFCRPCFYLNWQDIPFLVQCSECTKSTEQINLKTNIHLKKMASLARKVSLWLFLSSEEQMCGTHRETKKIFCEVDRSLLCLLCSSSQEHRYHRHRPIEWAAEEHREKLLQKMQSLWEKACENHRNLNVETTRTRCWKDYVNLRLEAIRAEYQKMPAFHHEEEKHNLEMLKKKGKEIFHRLHLSKAKMAHRMEILRGMYEELNEMCHKPDVELLQAFGDILHRSESVLLHMPQPLNPELSAGPITGLRDRLNQFRVHITLHHEEANSDIFLYEILRSMCIGCDHQDVPYFTATPRSFLAWGVQTFTSGKYYWEVHVGDSWNWAFGVCNMYRKEKNQNEKIDGKEGLFLLGCIKNDIQCSLFTTSPLMLQYIPKPTSRVGLFLDCEAKTVSFVDVNQSSLIYTIPNCSFSPPLRPIFCCIHF",
# TR49D_HUMAN
"MNSGISQVFQRELTCPICLNYFIDPVTIDCGHSFCRPCFYLNWQDIPILTQCFECLKTTQQRNLKTNIRLKKMASRARKASLWLFLSSEEQMCGTHRETKKIFCEVDRSLLCLLCSSSLEHRYHRHCPAEWAAEEHREKLLKKMQSLWEKVCENQRNLNVETTRISHWKDYVNVRLEAIRAEYQKMPAFHHEEEKHNLEMLKKKGKDIFHRLHLSKAKMAHRREILRGTYAELMKMCHKPDVELLQAFGDILHRSESVLLHMPQPLNLELRAGPITGLRDRLNQFRVDITLPHNEANSHIFRRGDLRSICIGCDRQNAPHITATPTSFLAWGAQTFTSGKYYWEVHVGDSWNWAFGVCNKYWKGTNQNGNIHGEEGLFSLGCVKNDIQCSLFTTSPLTLQYVPRPTNHVGLFLDCEARTVSFVDVNQSSPIHTIPNCSFSPPLRPIFCCVHL",
# TRIM5_HUMAN
"MASGILVNVKEEVTCPICLELLTQPLSLDCGHSFCQACLTANHKKSMLDKGESSCPVCRISYQPENIRPNRHVANIVEKLREVKLSPEGQKVDHCARHGEKLLLFCQEDGKVICWLCERSQEHRGHHTFLTEEVAREYQVKLQAALEMLRQKQQEAEELEADIREEKASWKTQIQYDKTNVLADFEQLRDILDWEESNELQNLEKEEEDILKSLTNSETEMVQQTQSLRELISDLEHRLQGSVMELLQGVDGVIKRTENVTLKKPETFPKNQRRVFRAPDLKGMLEVFRELTDVRRYWVDVTVAPNNISCAVISEDKRQVSSPKPQIIYGARGTRYQTFVNFNYCTGILGSQSITSGKHYWEVDVSKKTAWILGVCAGFQPDAMCNIEKNENYQPKYGYWVIGLEEGVKCSAFQDSSFHTPSVPFIVPLSVIICPDRVGVFLDYEACTVSFFNITNHGFLIYKFSHCSFSQPVFPYLNPRKCGVPMTLCSPSS",
# TRI50_HUMAN
"MAWQVSLPELEDRLQCPICLEVFKEPLMLQCGHSYCKGCLVSLSCHLDAELRCPVCRQAVDGSSSLPNVSLARVIEALRLPGDPEPKVCVHHRNPLSLFCEKDQELICGLCGLLGSHQHHPVTPVSTVYSRMKEELAALISELKQEQKKVDELIAKLVNNRTRIVNESDVFSWVIRREFQELHHLVDEEKARCLEGIGGHTRGLVASLDMQLEQAQGTRERLAQAECVLEQFGNEDHHKFIRKFHSMASRAEMPQARPLEGAFSPISFKPGLHQADIKLTVWKRLFRKVLPAPEPLKLDPATAHPLLELSKGNTVVQCGLLAQRRASQPERFDYSTCVLASRGFSCGRHYWEVVVGSKSDWRLGVIKGTASRKGKLNRSPEHGVWLIGLKEGRVYEAFACPRVPLPVAGHPHRIGLYLHYEQGELTFFDADRPDDLRPLYTFQADFQGKLYPILDTCWHERGSNSLPMVLPPPSGPGPLSPEQPTKL",
# TRI51_HUMAN
"MNSGILQVFQRALTCPICMNYFLDPVTIDCGHSFCRPCLYLNWQDTAVLAQCSECKKTTRQRNLNTDICLKNMAFIARKASLRQFLSSEEQICGMHRETKKMFCEVDKSLLCLPCSNSQEHRNHIHCPIEWAAEERREELLKKMQSLWEKACENLRNLNMETTRTRCWKDYVSLRIEAIRAEYQKMPAFLHEEEQHHLERLRKEGEDIFQQLNESKARMEHSRELLRGMYEDLKQMCHKADVELLQAFGDILHRYESLLLQVSEPVNPELSAGPITGLLDSLSGFRVDFTLQPERANSHIFLCGDLRSMNVGCDPQDDPDITGKSECFLVWGAQAFTSGKYYWEVHMGDSWNWAFGVCNNYWKEKRQNDKIDGEEGLFLLGCVKEDTHCSLFTTSPLVVQYVPRPTSTVGLFLDCEGRTVSFVDVDQSSLIYTIPNCSFSPPLRPIFCCSHF",
# TRI52_HUMAN
"MAGYATTPSPMQTLQEEAVCAICLDYFKDPVSISCGHNFCRGCVTQLWSKEDEEDQNEEEDEWEEEEDEEAVGAMDGWDGSIREVLYRGNADEELFQDQDDDELWLGDSGITNWDNVDYMWDEEEEEEEEDQDYYLGGLRPDLRIDVYREEEILEAYDEDEDEELYPDIHPPPSLPLPGQFTCPQCRKSFTRRSFRPNLQLANMVQIIRQMCPTPYRGNRSNDQGMCFKHQEALKLFCEVDKEAICVVCRESRSHKQHSVLPLEEVVQEYQEIKLETTLVGILQIEQESIHSKAYNQ",
# TRI54_HUMAN
"MNFTVGFKPLLGDAHSMDNLEKQLICPICLEMFSKPVVILPCQHNLCRKCANDVFQASNPLWQSRGSTTVSSGGRFRCPSCRHEVVLDRHGVYGLQRNLLVENIIDIYKQESSRPLHSKAEQHLMCEEHEEEKINIYCLSCEVPTCSLCKVFGAHKDCEVAPLPTIYKRQKSELSDGIAMLVAGNDRVQAVITQMEEVCQTIEDNSRRQKQLLNQRFESLCAVLEERKGELLQALAREQEEKLQRVRGLIRQYGDHLEASSKLVESAIQSMEEPQMALYLQQAKELINKVGAMSKVELAGRPEPGYESMEQFTVRVEHVAEMLRTIDFQPGASGEEEEVAPDGEEGSAGPEEERPDGP",
# TRI55_HUMAN
"MSASLNYKSFSKEQQTMDNLEKQLICPICLEMFTKPVVILPCQHNLCRKCASDIFQASNPYLPTRGGTTMASGGRFRCPSCRHEVVLDRHGVYGLQRNLLVENIIDIYKQESTRPEKKSDQPMCEEHEEERINIYCLNCEVPTCSLCKVFGAHKDCQVAPLTHVFQRQKSELSDGIAILVGSNDRVQGVISQLEDTCKTIEECCRKQKQELCEKFDYLYGILEERKNEMTQVITRTQEEKLEHVRALIKKYSDHLENVSKLVESGIQFMDEPEMAVFLQNAKTLLKKISEASKAFQMEKIEHGYENMNHFTVNLNREEKIIREIDFYREDEDEEEEEGGEGEKEGEGEVGGEAVEVEEVENVQTEFPGEDENPEKASELSQVELQAAPGALPVSSPEPPPALPPAADAPVTQGEVVPTGSEQTTESETPVPAAAETADPLFYPSWYKGQTRKATTNPPCTPGSEGLGQIGPPGSEDSNVRKAEVAAAAASERAAVSGKETSAPAATSQIGFEAPPLQGQAAAPASGSGADSEPARHIFSFSWLNSLNE",
# TRI56_HUMAN
"MVSHGSSPSLLEALSSDFLACKICLEQLRAPKTLPCLHTYCQDCLAQLADGGRVRCPECRETVPVPPEGVASFKTNFFVNGLLDLVKARACGDLRAGKPACALCPLVGGTSTGGPATARCLDCADDLCQACADGHRCTRQTHTHRVVDLVGYRAGWYDEEARERQAAQCPQHPGEALRFLCQPCSQLLCRECRLDPHLDHPCLPLAEAVRARRPGLEGLLAGVDNNLVELEAARRVEKEALARLREQAARVGTQVEEAAEGVLRALLAQKQEVLGQLRAHVEAAEEAARERLAELEGREQVARAAAAFARRVLSLGREAEILSLEGAIAQRLRQLQGCPWAPGPAPCLLPQLELHPGLLDKNCHLLRLSFEEQQPQKDGGKDGAGTQGGEESQSRREDEPKTERQGGVQPQAGDGAQTPKEEKAQTTREEGAQTLEEDRAQTPHEDGGPQPHRGGRPNKKKKFKGRLKSISREPSPALGPNLDGSGLLPRPIFYCSFPTRMPGDKRSPRITGLCPFGPREILVADEQNRALKRFSLNGDYKGTVPVPEGCSPCSVAALQSAVAFSASARLYLINPNGEVQWRRALSLSQASHAVAALPSGDRVAVSVAGHVEVYNMEGSLATRFIPGGKASRGLRALVFLTTSPQGHFVGSDWQQNSVVICDGLGQVVGEYKGPGLHGCQPGSVSVDKKGYIFLTLREVNKVVILDPKGSLLGDFLTAYHGLEKPRVTTMVDGRYLVVSLSNGTIHIFRVRSPDS",
# TRI58_HUMAN
"MAWAPPGERLREDARCPVCLDFLQEPVSVDCGHSFCLRCISEFCEKSDGAQGGVYACPQCRGPFRPSGFRPNRQLAGLVESVRRLGLGAGPGARRCARHGEDLSRFCEEDEAALCWVCDAGPEHRTHRTAPLQEAAGSYQVKLQMALELMRKELEDALTQEANVGKKTVIWKEKVEMQRQRFRLEFEKHRGFLAQEEQRQLRRLEAEERATLQRLRESKSRLVQQSKALKELADELQERCQRPALGLLEGVRGVLSRSKAVTRLEAENIPMELKTACCIPGRRELLRKFQVDVKLDPATAHPSLLLTADLRSVQDGEPWRDVPNNPERFDTWPCILGLQSFSSGRHYWEVLVGEGAEWGLGVCQDTLPRKGETTPSPENGVWALWLLKGNEYMVLASPSVPLLQLESPRCIGIFLDYEAGEISFYNVTDGSYIYTFNQLFSGLLRPYFFICDATPLILPPTTIAGSGNWASRDHLDPASDVRDDHL",
# TRI59_HUMAN
"MHNFEEELTCPICYSIFEDPRVLPCSHTFCRNCLENILQASGNFYIWRPLRIPLKCPNCRSITEIAPTGIESLPVNFALRAIIEKYQQEDHPDIVTCPEHYRQPLNVYCLLDKKLVCGHCLTIGQHHGHPIDDLQSAYLKEKDTPQKLLEQLTDTHWTDLTHLIEKLKEQKSHSEKMIQGDKEAVLQYFKELNDTLEQKKKSFLTALCDVGNLINQEYTPQIERMKEIREQQLELMALTISLQEESPLKFLEKVDDVRQHVQILKQRPLPEVQPVEIYPRVSKILKEEWSRTEIGQIKNVLIPKMKISPKRMSCSWPGKDEKEVEFLKILNIVVVTLISVILMSILFFNQHIITFLSEITLIWFSEASLSVYQSLSNSLHKVKNILCHIFYLLKEFVWKIVSH",
# TRIM6_HUMAN
"MTSPVLVDIREEVTCPICLELLTEPLSIDCGHSFCQACITPNGRESVIGQEGERSCPVCQTSYQPGNLRPNRHLANIVRRLREVVLGPGKQLKAVLCADHGEKLQLFCQEDGKVICWLCERSQEHRGHHTFLVEEVAQEYQEKFQESLKKLKNEEQEAEKLTAFIREKKTSWKNQMEPERCRIQTEFNQLRNILDRVEQRELKKLEQEEKKGLRIIEEAENDLVHQTQSLRELISDLERRCQGSTMELLQDVSDVTERSEFWTLRKPEALPTKLRSMFRAPDLKRMLRVCRELTDVQSYWVDVTLNPHTANLNLVLAKNRRQVRFVGAKVSGPSCLEKHYDCSVLGSQHFSSGKHYWEVDVAKKTAWILGVCSNSLGPTFSFNHFAQNHSAYSRYQPQSGYWVIGLQHNHEYRAYEDSSPSLLLSMTVPPRRVGVFLDYEAGTVSFYNVTNHGFPIYTFSKYYFPTTLCPYFNPCNCVIPMTLRRPSS",
# TRI60_HUMAN
"MEFVTALVNLQEESSCPICLEYLKDPVTINCGHNFCRSCLSVSWKDLDDTFPCPVCRFCFPYKSFRRNPQLRNLTEIAKQLQIRRSKRKRQKENAMCEKHNQFLTLFCVKDLEILCTQCSFSTKHQKHYICPIKKAASYHREILEGSLEPLRNNIERVEKVIILQGSKSVELKKKVEYKREEINSEFEQIRLFLQNEQEMILRQIQDEEMNILAKLNENLVELSDYVSTLKHLLREVEGKSVQSNLELLTQAKSMHHKYQNLKCPELFSFRLTKYGFSLPPQYSGLDRIIKPFQVDVILDLNTAHPQLLVSEDRKAVRYERKKRNICYDPRRFYVCPAVLGSQRFSSGRHYWEVEVGNKPKWILGVCQDCLLRNWQDQPSVLGGFWAIGRYMKSGYVASGPKTTQLLPVVKPSKIGIFLDYELGDLSFYNMNDRSILYTFNDCFTEAVWPYFYTGTDSEPLKICSVSDSER",
# TRI61_HUMAN
"MEFVTALADLRAEASCPICLDYLKDPVTISCGHNFCLSCIIMSWKDLHDSFPCPFCHFCCPERKFISNPQLGSLTEIAKQLQIRSKKRKRQEEKHVCKKHNQVLTFFCQKDLELLCPRCSLSTDHQHHCVWPIKKAASYHRKKLEEYNAPWKERVELIEKVITMQTRKSLELKKKMESPSVTRLECSCTISAHFNLRLPGSSDSSASGS",
# TRI62_HUMAN
"MACSLKDELLCSICLSIYQDPVSLGCEHYFCRRCITEHWVRQEAQGARDCPECRRTFAEPALAPSLKLANIVERYSSFPLDAILNARRAARPCQAHDKVKLFCLTDRALLCFFCDEPALHEQHQVTGIDDAFDELQRELKDQLQALQDSEREHTEALQLLKRQLAETKSSTKSLRTTIGEAFERLHRLLRERQKAMLEELEADTARTLTDIEQKVQRYSQQLRKVQEGAQILQERLAETDRHTFLAGVASLSERLKGKIHETNLTYEDFPTSKYTGPLQYTIWKSLFQDIHPVPAALTLDPGTAHQRLILSDDCTIVAYGNLHPQPLQDSPKRFDVEVSVLGSEAFSSGVHYWEVVVAEKTQWVIGLAHEAASRKGSIQIQPSRGFYCIVMHDGNQYSACTEPWTRLNVRDKLDKVGVFLDYDQGLLIFYNADDMSWLYTFREKFPGKLCSYFSPGQSHANGKNVQPLRINTVRI",
# TRI63_HUMAN
"MDYKSSLIQDGNPMENLEKQLICPICLEMFTKPVVILPCQHNLCRKCANDIFQAANPYWTSRGSSVSMSGGRFRCPTCRHEVIMDRHGVYGLQRNLLVENIIDIYKQECSSRPLQKGSHPMCKEHEDEKINIYCLTCEVPTCSMCKVFGIHKACEVAPLQSVFQGQKTELNNCISMLVAGNDRVQTIITQLEDSRRVTKENSHQVKEELSQKFDTLYAILDEKKSELLQRITQEQEKKLSFIEALIQQYQEQLDKSTKLVETAIQSLDEPGGATFLLTAKQLIKSIVEASKGCQLGKTEQGFENMDFFTLDLEHIADALRAIDFGTDEEEEEFIEEEDQEEEESTEGKEEGHQ",
# TRI64_HUMAN
"MDSDDLQVFQNELICCICVNYFIDPVTIDCGHSFCRPCLCLCSEEGRAPMRCPSCRKISEKPNFNTNVVLKKLSSLARQTRPQNINSSDNICVLHEETKELFCEADKRLLCGPCSESPEHMAHSHSPIGWAAEECREKLIKEMDYLWEINQETRNNLNQETRTFHSLKDYVSVRKRIITIQYQKMPIFLDEEEQRHLQALEREAEELFQQLQDSQVRMTQHLERMKDMYRELWETCHVPDVELLQDVRNVSARTDLAQMQKPQPVNPELTSWCITGVLDMLNNFRVDSALSTEMIPCYISLSEDVRYVIFGDDHLSAPTDPQGVDSFAVWGAQAFTSGKHYWEVDVTLSSNWILGVCQDSRTADANFVIDSDERFFLISSKRSNHYSLSTNSPPLIQYVQRPLGQVGVFLDYDNGSVSFFDVSKGSLIYGFPPSSFSSPLRPFFCFGCT",
# TR64B_HUMAN
"MDSDDLQVFQNELICCICVNYFIDPVTIDCGHSFCRPCLCLCSEEGRAPMRCPSCRKTSEKPNFNTNLVLKKLSSLARQTRPQNINSSDNICVLHEETKELFCEADKRLLCGPCSESPEHMAHSHSPIGWAAEECREKLIKEMDYLWEINQETRNNLNQETSTFHSLKDYVSVRKRIITIQYQKMPIFLDEEEQRHLQALEREAEELFQQLQDSQVRMTQHLERMKDMYRELWETCHMPDVVLLQDVRNVSARTDLAQMQKPQPVNPELTSWCITGVLDMLNNFRVDSALSTEMIPCYISLSEDVRYVIFGDDHLSAPTDPQGVDSFAVWGAQAFTSGKHYWEVDVTLSSNWILGVCRDSRTADANFVIDSDERFFLISSKRSNHYSLSTNSPPLIQYVQRPLGRVGVFLDYDNGSVSFFDVSKGSLIYGFPPSSFSSPLRPFFCFGCT",
# TR64C_HUMAN
"MDSDTLRVFQNELICCICVNYFIDPVTTDCVHSFCRPCLCLCSEEGRAPMRCPLCRKISEKPNFNTNVALKKLASLARQTRPQNINSSDNICVLHEETKELFCEADKRLLCGPCSESPEHMAHSHSPIGWAAEECRVQKLIKEMDYLWKINQETQNNLNQETSKFCSLVDYVSLRKVIITIQYQKMHIFLDEEEQRHLQALEREAKELFQQLQDSQVRMTQHLEGMKDMYRELWETYHMPDVELLQDVGNISARTDLAQMPKPQPVNPELTSWCITGVLDMLNNFRVDNALSTEMTPCYISLSEDVRRVIFGDDHRSAPMDPQGVESFAVWCAQAFTSGKHYWEVDVTHSSNWILGVCRDSRTADTNIVIDSDKTFFSISSKTSNHYSLSTNSPPLIQYVQRPLGWVGVFLDYDNGSVSFFDVSKGSLIYGFPPSSFSSPLRPFFCFGCT",
# TRI65_HUMAN
"MAAQLLEEKLTCAICLGLYQDPVTLPCGHNFCGACIRDWWDRCGKACPECREPFPDGAELRRNVALSGVLEVVRAGPARDPGPDPGPGPDPAARCPRHGRPLELFCRTEGRCVCSVCTVRECRLHERALLDAERLKREAQLRASLEVTQQQATQAEGQLLELRKQSSQIQNSACILASWVSGKFSSLLQALEIQHTTALRSIEVAKTQALAQARDEEQRLRVHLEAVARHGCRIRELLEQVDEQTFLQESQLLQPPGPLGPLTPLQWDEDQQLGDLKQLLSRLCGLLLEEGSHPGAPAKPVDLAPVEAPGPLAPVPSTVCPLRRKLWQNYRNLTFDPVSANRHFYLSRQDQQVKHCRQSRGPGGPGSFELWQVQCAQSFQAGHHYWEVRASDHSVTLGVSYPQLPRCRLGPHTDNIGRGPCSWGLCVQEDSLQAWHNGEAQRLPGVSGRLLGMDLDLASGCLTFYSLEPQTQPLYTFHALFNQPLTPVFWLLEGRTLTLCHQPGAVFPLGPQEEVLS",
# TRI67_HUMAN
"MEEELKCPVCGSLFREPIILPCSHNVCLPCARTIAVQTPDGEQHLPQPLLLSRGSGLQAGAAAAASLEHDAAAGPACGGAGGSAAGGLGGGAGGGGDHADKLSLYSETDSGYGSYTPSLKSPNGVRVLPMVPAPPGSSAAAARGAACSSLSSSSSSITCPQCHRSASLDHRGLRGFQRNRLLEAIVQRYQQGRGAVPGTSAAAAVAICQLCDRTPPEPAATLCEQCDVLYCSACQLKCHPSRGPFAKHRLVQPPPPPPPPAEAASGPTGTAQGAPSGGGGCKSPGGAGAGATGGSTARKFPTCPEHEMENYSMYCVSCRTPVCYLCLEEGRHAKHEVKPLGAMWKQHKAQLSQALNGVSDKAKEAKEFLVQLKNILQQIQENGLDYEACLVAQCDALVDALTRQKAKLLTKVTKEREHKLKMVWDQINHCTLKLRQSTGLMEYCLEVIKENDPSGFLQISDALIKRVQVSQEQWVKGALEPKVSAEFDLTLDSEPLLQAIHQLDFIQMKCRVPPVPLLQLEKCCTRNNSVTLAWRMPPFTHSPVDGYILELDDGAGGQFREVYVGKETLCTIDGLHFNSTYNARVKAFNSSGVGPYSKTVVLQTSDVAWFTFDPNSGHRDIILSNDNQTATCSSYDDRVVLGTAAFSKGVHYWELHVDRYDNHPDPAFGVARASVVKDMMLGKDDKAWAMYVDNNRSWFMHCNSHTNRTEGGVCKGATVGVLLDLNKHTLTFFINGQQQGPTAFSHVDGVFMPALSLNRNVQVTLHTGLEVPTNLGRPKLSGN",
# TRI68_HUMAN
"MDPTALVEAIVEEVACPICMTFLREPMSIDCGHSFCHSCLSGLWEIPGESQNWGYTCPLCRAPVQPRNLRPNWQLANVVEKVRLLRLHPGMGLKGDLCERHGEKLKMFCKEDVLIMCEACSQSPEHEAHSVVPMEDVAWEYKWELHEALEHLKKEQEEAWKLEVGERKRTATWKIQVETRKQSIVWEFEKYQRLLEKKQPPHRQLGAEVAAALASLQREAAETMQKLELNHSELIQQSQVLWRMIAELKERSQRPVRWMLQDIQEVLNRSKSWSLQQPEPISLELKTDCRVLGLREILKTYAADVRLDPDTAYSRLIVSEDRKRVHYGDTNQKLPDNPERFYRYNIVLGSQCISSGRHYWEVEVGDRSEWGLGVCKQNVDRKEVVYLSPHYGFWVIRLRKGNEYRAGTDEYPILSLPVPPRRVGIFVDYEAHDISFYNVTDCGSHIFTFPRYPFPGRLLPYFSPCYSIGTNNTAPLAICSLDGED",
# TRI69_HUMAN
"MEVSTNPSSNIDPGDYVEMNDSITHLPSKVVIQDITMELHCPLCNDWFRDPLMLSCGHNFCEACIQDFWRLQAKETFCPECKMLCQYNNCTFNPVLDKLVEKIKKLPLLKGHPQCPEHGENLKLFSKPDGKLICFQCKDARLSVGQSKEFLQISDAVHFFTEELAIQQGQLETTLKELQTLRNMQKEAIAAHKENKLHLQQHVSMEFLKLHQFLHSKEKDILTELREEGKALNEEMELNLSQLQEQCLLAKDMLVSIQAKTEQQNSFDFLKDITTLLHSLEQGMKVLATRELISRKLNLGQYKGPIQYMVWREMQDTLCPGLSPLTLDPKTAHPNLVLSKSQTSVWHGDIKKIMPDDPERFDSSVAVLGSRGFTSGKWYWEVEVAKKTKWTVGVVRESIIRKGSCPLTPEQGFWLLRLRNQTDLKALDLPSFSLTLTNNLDKVGIYLDYEGGQLSFYNAKTMTHIYTFSNTFMEKLYPYFCPCLNDGGENKEPLHILHPQ",
# TRIM7_HUMAN
"MAAVGPRTGPGTGAEALALAAELQGEATCSICLELFREPVSVECGHSFCRACIGRCWERPGAGSVGAATRAPPFPLPCPQCREPARPSQLRPNRQLAAVATLLRRFSLPAAAPGEHGSQAAAARAAAARCGQHGEPFKLYCQDDGRAICVVCDRAREHREHAVLPLDEAVQEAKELLESRLRVLKKELEDCEVFRSTEKKESKELLKQMAAEQEKVGAEFQALRAFLVEQEGRLLGRLEELSREVAQKQNENLAQLGVEITQLSKLSSQIQETAQKPDLDFLQEFKSTLSRCSNVPGPKPTTVSSEMKNKVWNVSLKTFVLKGMLKKFKEDLRGELEKEEKVELTLDPDTANPRLILSLDLKGVRLGERAQDLPNHPCRFDTNTRVLASCGFSSGRHHWEVEVGSKDGWAFGVARESVRRKGLTPFTPEEGVWALQLNGGQYWAVTSPERSPLSCGHLSRVRVALDLEVGAVSFYAVEDMRHLYTFRVNFQERVFPLFSVCSTGTYLRIWP",
# LIN41_HUMAN
"MASFPETDFQICLLCKEMCGSPAPLSSNSSASSSSSQTSTSSGGGGGGPGAAARRLHVLPCLHAFCRPCLEAHRLPAAGGGAAGEPLKLRCPVCDQKVVLAEAAGMDALPSSAFLLSNLLDAVVATADEPPPKNGRAGAPAGAGGHSNHRHHAHHAHPRASASAPPLPQAPQPPAPSRSAPGGPAASPSALLLRRPHGCSSCDEGNAASSRCLDCQEHLCDNCVRAHQRVRLTKDHYIERGPPGPGAAAAAQQLGLGPPFPGPPFSILSVFPERLGFCQHHDDEVLHLYCDTCSVPICRECTMGRHGGHSFIYLQEALQDSRALTIQLLADAQQGRQAIQLSIEQAQTVAEQVEMKAKVVQSEVKAVTARHKKALEERECELLWKVEKIRQVKAKSLYLQVEKLRQNLNKLESTISAVQQVLEEGRALDILLARDRMLAQVQELKTVRSLLQPQEDDRVMFTPPDQALYLAIKSFGFVSSGAFAPLTKATGDGLKRALQGKVASFTVIGYDHDGEPRLSGGDLMSAVVLGPDGNLFGAEVSDQQNGTYVVSYRPQLEGEHLVSVTLCNQHIENSPFKVVVKSGRSYVGIGLPGLSFGSEGDSDGKLCRPWGVSVDKEGYIIVADRSNNRIQVFKPCGAFHHKFGTLGSRPGQFDRPAGVACDASRRIVVADKDNHRIQIFTFEGQFLLKFGEKGTKNGQFNYPWDVAVNSEGKILVSDTRNHRIQLFGPDGVFLNKYGFEGALWKHFDSPRGVAFNHEGHLVVTDFNNHRLLVIHPDCQSARFLGSEGTGNGQFLRPQGVAVDQEGRIIVADSRNHRVQMFESNGSFLCKFGAQGSGFGQMDRPSGIAITPDGMIVVVDFGNNRILVF",
# TRI72_HUMAN
"MSAAPGLLHQELSCPLCLQLFDAPVTAECGHSFCRACLGRVAGEPAADGTVLCPCCQAPTRPQALSTNLQLARLVEGLAQVPQGHCEEHLDPLSIYCEQDRALVCGVCASLGSHRGHRLLPAAEAHARLKTQLPQQKLQLQEACMRKEKSVAVLEHQLVEVEETVRQFRGAVGEQLGKMRVFLAALEGSLDREAERVRGEAGVALRRELGSLNSYLEQLRQMEKVLEEVADKPQTEFLMKYCLVTSRLQKILAESPPPARLDIQLPIISDDFKFQVWRKMFRALMPALEELTFDPSSAHPSLVVSSSGRRVECSEQKAPPAGEDPRQFDKAVAVVAHQQLSEGEHYWEVDVGDKPRWALGVIAAEAPRRGRLHAVPSQGLWLLGLREGKILEAHVEAKEPRALRSPERRPTRIGLYLSFGDGVLSFYDASDADALVPLFAFHERLPRPVYPFFDVCWHDKGKNAQPLLLVGPEGAEA",
# TRI73_HUMAN
"MAWQVSLLELEDRLQCPICLEVFKESLMLQCGHSYCKGCLVSLSYHLDTKVRCPMCWQVVDGSSSLPNVSLAWVIEALRLPGDPEPKVCVHHRNPLSLFCEKDQELICGLCGLLGSHQHHPVTPVSTVCSRMKEELAALFSELKQEQKKVDELIAKLVKNRTRIVNESDVFSWVIRREFQELRHPVDEEKARCLEGIGGHTRGLVASLDMQLEQAQGTRERLAQAECVLEQFGNEDHHEFIWKFHSMASR",
# TRI74_HUMAN
"MAWQVSLLELEDWLQCPICLEVFKESLMLQCGHSYCKGCLVSLSYHLDTKVRCPMCWQVVDGSSSLPNVSLAWVIEALRLPGDPEPKVCVHHRNPLSLFCEKDQELICGLCGLLGSHQHHPVTPVSTVCSRMKEELAALFSELKQEQKKVDELIAKLVKNRTRIVNESDVFSWVIRREFQELRHPVDEEKARCLEGIGGHTRGLVASLDMQLEQAQGTRERLAQAECVLEQFGNEDHHEFIWKFHSMASR",
# TRI75_HUMAN
"MAVAAALTGLQAEAKCSICLDYLSDPVTIECGHNFCRSCIQQSWLDLQELFPCPVCRHQCQEGHFRSNTQLGRMIEIAKLLQSTKSNKRKQEETTLCEKHNQPLSVFCKEDLMVLCPLCTQPPDHQGHHVRPIEKAAIHYRKRFCSYIQPLKKQLADLQKLISTQSKKPLELREMVENQRQELSSEFEHLNQFLDREQQAVLSRLAEEEKDNQQKLSANITAFSNYSATLKSQLSKVVELSELSELELLSQIKIFYESENESSPSIFSIHLKRDGCSFPPQYSALQRIIKKFKVEIILDPETAHPNLIVSEDKKRVRFTKRKQKVPGFPKRFTVKPVVLGFPYFHSGRHFWEIEVGDKSEWAIGICKDSLPTKARRPSSAQQECWRIELQDDGYHAPGAFPTPLLLEVKARAIGIFLDYEMGEISFYNMAEKSHICTFTDTFTGPLRPYFYVGPDSQPLRICTGTVCE",
# TRI77_HUMAN
"MASAITQCSTSELTCSICTDYLTDPVTICCGHRFCSPCLCLLWEDTLTPNCCPVCREISQQMYFKRIIFAEKQVIPTRESVPCQLSSSAMLICRRHQEIKNLICETDRSLLCFLCSQSPRHATHKHYMTREADEYYRKKLLIQMKSIWKKKQKNQRNLNRETNIIGTWEVFINLRSMMISAEYPKVCQYLREEEQKHVESLAREGRIIFQQLKRSQTRMAKMGILLREMYEKLKEMSCKADVNLPQDLGDVMKRNEFLRLAMPQPVNPQLSAWTITGVSERLNFFRVYITLDRKICSNHKLLFEDLRHLQCSLDDTDMSCNPTSTQYTSSWGAQILSSGKHYWEVDVKDSCNWVIGLCREAWTKRNDMRLDSEGIFLLLCLKVDDHFSLFSTSPLLPHYIPRPQGWLGVFLDYECGIVSFVNVAQSSLICSFLSRIFYFPLRPFICHGSK",
# TRIM8_HUMAN
"MAENWKNCFEEELICPICLHVFVEPVQLPCKHNFCRGCIGEAWAKDSGLVRCPECNQAYNQKPGLEKNLKLTNIVEKFNALHVEKPPAALHCVFCRRGPPLPAQKVCLRCEAPCCQSHVQTHLQQPSTARGHLLVEADDVRAWSCPQHNAYRLYHCEAEQVAVCQYCCYYSGAHQGHSVCDVEIRRNEIRKMLMKQQDRLEEREQDIEDQLYKLESDKRLVEEKVNQLKEEVRLQYEKLHQLLDEDLRQTVEVLDKAQAKFCSENAAQALHLGERMQEAKKLLGSLQLLFDKTEDVSFMKNTKSVKILMDRTQTCTSSSLSPTKIGHLNSKLFLNEVAKKEKQLRKMLEGPFSTPVPFLQSVPLYPCGVSSSGAEKRKHSTAFPEASFLETSSGPVGGQYGAAGTASGEGQSGQPLGPCSSTQHLVALPGGAQPVHSSPVFPPSQYPNGSAAQQPMLPQYGGRKILVCSVDNCYCSSVANHGGHQPYPRSGHFPWTVPSQEYSHPLPPTPSVPQSLPSLAVRDWLDASQQPGHQDFYRVYGQPSTKHYVTS",
# TRIM9_HUMAN
"MEEMEEELKCPVCGSFYREPIILPCSHNLCQACARNILVQTPESESPQSHRAAGSGVSDYDYLDLDKMSLYSEADSGYGSYGGFASAPTTPCQKSPNGVRVFPPAMPPPATHLSPALAPVPRNSCITCPQCHRSLILDDRGLRGFPKNRVLEGVIDRYQQSKAAALKCQLCEKAPKEATVMCEQCDVFYCDPCRLRCHPPRGPLAKHRLVPPAQGRVSRRLSPRKVSTCTDHELENHSMYCVQCKMPVCYQCLEEGKHSSHEVKALGAMWKLHKSQLSQALNGLSDRAKEAKEFLVQLRNMVQQIQENSVEFEACLVAQCDALIDALNRRKAQLLARVNKEHEHKLKVVRDQISHCTVKLRQTTGLMEYCLEVIKENDPSGFLQISDALIRRVHLTEDQWGKGTLTPRMTTDFDLSLDNSPLLQSIHQLDFVQVKASSPVPATPILQLEECCTHNNSATLSWKQPPLSTVPADGYILELDDGNGGQFREVYVGKETMCTVDGLHFNSTYNARVKAFNKTGVSPYSKTLVLQTSEVAWFAFDPGSAHSDIILSNDNLTVTCSSYDDRVVLGKTGFSKGIHYWELTVDRYDNHPDPAFGVARMDVMKDVMLGKDDKAWAMYVDNNRSWFMHNNSHTNRTEGGITKGATIGVLLDLNRKNLTFFINDEQQGPIAFDNVEGLFFPAVSLNRNVQVTLHTGLPVPDFYSSRASIA",
# TRIML_HUMAN
"MSTADLMENLREELTCFICLDYFSSPVTTECGHSFCLVCLLRSWEEHNTPLSCPECWRTLEGPHFQSNERLGRLASIARQLRSQVLQSEDEQGSYGRMPTTAKALSDDEQGGSAFVAQSHGANRVHLSSEAEEHHREKLQEILNLLRVRRKEAQAVLTHEKERVKLCQEETKTCKQVVVSEYMKMHQFLKEEEQLQLQLLEQEEKENMRKLRNNEIKLTQQIRSLSKMIAQIESSSQSSAFESLEEVRGALERSEPLLLQCPEATTTELSLCRITGMKEMLRKFSTEITLDPATANAYLVLSEDLKSVKYGGSRQQLPDNPERFDQSATVLGTQIFTSGRHYWEVEVGNKTEWEVGICKDSVSRKGNLPKPPGDLFSLIGLKIGDDYSLWVSSPLKGQHVREPVCKVGVFLDYESGHIAFYNGTDESLIYSFPQASFQEALRPIFSPCLPNEGTNTDPLTICSLNSHV",
# TRIMM_HUMAN
"MSKRLSPQLQHNITEDAYCETHLEPTRLFCDVDQITLCSKCFQSQEHKHHMVCGIQEAAENYRKLFQEILNTSREKLEAAKSILTDEQERMAMIQEEEQNFKKMIESEYSMRLRLLNEECEQNLQRQQECISDLNLRETLLNQAIKLATELEEMFQEMLQRLGRVGRENMEKLKESEARASEQVRSLLKLIVELEKKCGEGTLALLKNAKYSLERSKSLLLEHLEPAHITDLSLCHIRGLSSMFRVLQRHLTLDPETAHPCLALSEDLRTMRLRHGQQDGAGNPERLDFSAMVLAAESFTSGRHYWEVDVEKATRWQVGIYHGSADAKGSTARASGEKVLLTGSVMGTEWTLWVFPPLKRLFLEKKLDTVGVFLDCEHGQISFYNVTEMSLIYNFSHCAFQGALRPVFSLCIPNGDTSPDSLTILQHGPSCDATVSP",
# TRIPC_HUMAN
"MSNRPNNNPGGSLRRSQRNTAGAQPQDDSIGGRSCSSSSAVIVPQPEDPDRANTSERQKTGQVPKKDNSRGVKRSASPDYNRTNSPSSAKKPKALQHTESPSETNKPHSKSKKRHLDQEQQLKSAQSPSTSKAHTRKSGATGGSRSQKRKRTESSCVKSGSGSESTGAEERSAKPTKLASKSATSAKAGCSTITDSSSAASTSSSSSAVASASSTVPPGARVKQGKDQNKARRSRSASSPSPRRSSREKEQSKTGGSSKFDWAARFSPKVSLPKTKLSLPGSSKSETSKPGPSGLQAKLASLRKSTKKRSESPPAELPSLRRSTRQKTTGSCASTSRRGSGLGKRGAAEARRQEKMADPESNQEAVNSSAARTDEAPQGAAGAVGMTTSGESESDDSEMGRLQALLEARGLPPHLFGPLGPRMSQLFHRTIGSGASSKAQQLLQGLQASDESQQLQAVIEMCQLLVMGNEETLGGFPVKSVVPALITLLQMEHNFDIMNHACRALTYMMEALPRSSAVVVDAIPVFLEKLQVIQCIDVAEQALTALEMLSRRHSKAILQAGGLADCLLYLEFFSINAQRNALAIAANCCQSITPDEFHFVADSLPLLTQRLTHQDKKSVESTCLCFARLVDNFQHEENLLQQVASKDLLTNVQQLLVVTPPILSSGMFIMVVRMFSLMCSNCPTLAVQLMKQNIAETLHFLLCGASNGSCQEQIDLVPRSPQELYELTSLICELMPCLPKEGIFAVDTMLKKGNAQNTDGAIWQWRDDRGLWHPYNRIDSRIIEQINEDTGTARAIQRKPNPLANSNTSGYSESKKDDARAQLMKEDPELAKSFIKTLFGVLYEVYSSSAGPAVRHKCLRAILRIIYFADAELLKDVLKNHAVSSHIASMLSSQDLKIVVGALQMAEILMQKLPDIFSVYFRREGVMHQVKHLAESESLLTSPPKACTNGSGSMGSTTSVSSGTATAATHAAADLGSPSLQHSRDDSLDLSPQGRLSDVLKRKRLPKRGPRRPKYSPPRDDDKVDNQAKSPTTTQSPKSSFLASLNPKTWGRLSTQSNSNNIEPARTAGGSGLARAASKDTISNNREKIKGWIKEQAHKFVERYFSSENMDGSNPALNVLQRLCAATEQLNLQVDGGAECLVEIRSIVSESDVSSFEIQHSGFVKQLLLYLTSKSEKDAVSREIRLKRFLHVFFSSPLPGEEPIGRVEPVGNAPLLALVHKMNNCLSQMEQFPVKVHDFPSGNGTGGSFSLNRGSQALKFFNTHQLKCQLQRHPDCANVKQWKGGPVKIDPLALVQAIERYLVVRGYGRVREDDEDSDDDGSDEEIDESLAAQFLNSGNVRHRLQFYIGEHLLPYNMTVYQAVRQFSIQAEDERESTDDESNPLGRAGIWTKTHTIWYKPVREDEESNKDCVGGKRGRAQTAPTKTSPRNAKKHDELWHDGVCPSVSNPLEVYLIPTPPENITFEDPSLDVILLLRVLHAISRYWYYLYDNAMCKEIIPTSEFINSKLTAKANRQLQDPLVIMTGNIPTWLTELGKTCPFFFPFDTRQMLFYVTAFDRDRAMQRLLDTNPEINQSDSQDSRVAPRLDRKKRTVNREELLKQAESVMQDLGSSRAMLEIQYENEVGTGLGPTLEFYALVSQELQRADLGLWRGEEVTLSNPKGSQEGTKYIQNLQGLFALPFGRTAKPAHIAKVKMKFRFLGKLMAKAIMDFRLVDLPLGLPFYKWMLRQETSLTSHDLFDIDPVVARSVYHLEDIVRQKKRLEQDKSQTKESLQYALETLTMNGCSVEDLGLDFTLPGFPNIELKKGGKDIPVTIHNLEEYLRLVIFWALNEGVSRQFDSFRDGFESVFPLSHLQYFYPEELDQLLCGSKADTWDAKTLMECCRPDHGYTHDSRAVKFLFEILSSFDNEQQRLFLQFVTGSPRLPVGGFRSLNPPLTIVRKTFESTENPDDFLPSVMTCVNYLKLPDYSSIEIMREKLLIAAREGQQSFHLS",
# TTC3_HUMAN
"MDNFAEGDFTVADYALLEDCPHVDDCVFAAEFMSNDYVRVTQLYCDGVGVQYKDYIQSERNLEFDICSIWCSKPISVLQDYCDAIKINIFWPLLFQHQNSSVISRLHPCVDANNSRASEINLKKLQHLELMEDIVDLAKKVANDSFLIGGLLRIGCKIENKILAMEEALNWIKYAGDVTILTKLGSIDNCWPMLSIFFTEYKYHITKIVMEDCNLLEELKTQSCMDCIEEGELMKMKGNEEFSKERFDIAIIYYTRAIEYRPENYLLYGNRALCFLRTGQFRNALGDGKRATILKNTWPKGHYRYCDALSMLGEYDWALQANIKAQKLCKNDPEGIKDLIQQHVKLQKQIEDLQGRTANKDPIKAFYENRAYTPRSLSAPIFTTSLNFVEKERDFRKINHEMANGGNQNLKVADEALKVDDCDCHPEFSPPSSQPPKHKGKQKSRNNESEKFSSSSPLTLPADLKNILEKQFSKSSRAAHQDFANIMKMLRSLIQDGYMALLEQRCRSAAQAFTELLNGLDPQKIKQLNLAMINYVLVVYGLAISLLGIGQPEELSEAENQFKRIIEHYPSEGLDCLAYCGIGKVYLKKNRFLEALNHFEKARTLIYRLPGVLTWPTSNVIIEESQPQKIKMLLEKFVEECKFPPVPDAICCYQKCHGYSKIQIYITDPDFKGFIRISCCQYCKIEFHMNCWKKLKTTTFNDKIDKDFLQGICLTPDCEGVISKIIIFSSGGEVKCEFEHKVIKEKVPPRPILKQKCSSLEKLRLKEDKKLKRKIQKKEAKKLAQERMEEDLRESNPPKNEEQKETVDNVQRCQFLDDRILQCIKQYADKIKSGIQNTAMLLKELLSWKVLSTEDYTTCFSSRNFLNEAVDYVIRHLIQENNRVKTRIFLHVLSELKEVEPKLAAWIQKLNSFGLDATGTFFSRYGASLKLLDFSIMTFLWNEKYGHKLDSIEGKQLDYFSEPASLKEARCLIWLLEEHRDKFPALHSALDEFFDIMDSRCTVLRKQDSGEAPFSSTKVKNKSKKKKPKDSKPMLVGSGTTSVTSNNEIITSSEDHSNRNSDSAGPFAVPDHLRQDVEEFEALYDQHSNEYVVRNKKLWDMNPKQKCSTLYDYFSQFLEEHGPLDMSNKMFSAEYEFFPEETRQILEKAGGLKPFLLGCPRFVVIDNCIALKKVASRLKKKRKKKNIKTKVEEISKAGEYVRVKLQLNPAAREFKPDVKSKPVSDSSSAPAFENVKPKPVSANSPKPACEDVKAKPVSDNSSRQVSEDGQPKGVSSNSPKPGSEDANYKRVSCNSPKPVLEDVKPTYWAQSHLVTGYCTYLPFQRFDITQTPPAYINVLPGLPQYTSIYTPLASLSPEYQLPRSVPVVPSFVANDRADKNAAAYFEGHHLNAENVAGHQIASETQILEGSLGISVKSHCSTGDAHTVLSESNRNDEHCGNSNNKCEVIPESTSAVTNIPHVQMVAIQVSWNIIHQEVNTEPYNPFEERQGEISRIEKEHQVLQDQLQEVYENYEQIKLKGLEETRDLEEKLKRHLEENKISKTELDWFLQDLEREIKKWQQEKKEIQERLKSLKKKIKKVSNASEMYTQKNDGKEKEHELHLDQSLEISNTLTNEKMKIEEYIKKGKEDYEESHQRAVAAEVSVLENWKESEVYKLQIMESQAEAFLKKLGLISRDPAAYPDMESDIRSWELFLSNVTKEIEKAKSQFEEQIKAIKNGSRLSELSKVQISELSFPACNTVHPELLPESSGDDGQGLVTSASDVTGNHAALHRDPSVFSAGDSPGEAPSALLPGPPPGQPEATQLTGPKRAGQAALSERSPVADRKQPVPPGRAARSSQSPKKPFNSIIEHLSVVFPCYNSTELAGFIKKVRSKNKNSLSGLSIDEIVQRVTEHILDEQKKKKPNPGKDKRTYEPSSATPVTRSSQGSPSVVVAPSPKTKGQKAEDVPVRIALGASSCEICHEVFKSKNVRVLKCGHKYHKGCFKQWLKGQSACPACQGRDLLTEESPSGRGWPSQNQELPSCSSR",
# UBE3A_HUMAN
"MEKLHQCYWKSGEPQSDDIEASRMKRAAAKHLIERYYHQLTEGCGNEACTNEFCASCPTFLRMDNNAAAIKALELYKINAKLCDPHPSKKGASSAYLENSKGAPNNSCSEIKMNKKGARIDFKDVTYLTEEKVYEILELCREREDYSPLIRVIGRVFSSAEALVQSFRKVKQHTKEELKSLQAKDEDKDEDEKEKAACSAAAMEEDSEASSSRIGDSSQGDNNLQKLGPDDVSVDIDAIRRVYTRLLSNEKIETAFLNALVYLSPNVECDLTYHNVYSRDPNYLNLFIIVMENRNLHSPEYLEMALPLFCKAMSKLPLAAQGKLIRLWSKYNADQIRRMMETFQQLITYKVISNEFNSRNLVNDDDAIVAASKCLKMVYYANVVGGEVDTNHNEEDDEEPIPESSELTLQELLGEERRNKKGPRVDPLETELGVKTLDCRKPLIPFEEFINEPLNEVLEMDKDYTFFKVETENKFSFMTCPFILNAVTKNLGLYYDNRIRMYSERRITVLYSLVQGQQLNPYLRLKVRRDHIIDDALVRLEMIAMENPADLKKQLYVEFEGEQGVDEGGVSKEFFQLVVEEIFNPDIGMFTYDESTKLFWFNPSSFETEGQFTLIGIVLGLAIYNNCILDVHFPMVVYRKLMGKKGTFRDLGDSHPVLYQSLKDLLEYEGNVEDDMMITFQISQTDLFGNPMMYDLKENGDKIPITNENRKEFVNLYSDYILNKSVEKQFKAFRRGFHMVTNESPLKYLFRPEEIELLICGSRNLDFQALEETTEYDGGYTRDSVLIREFWEIVHSFTDEQKRLFLQFTTGTDRAPVGGLGKLKMIIAKNGPDTERLPTSHTCFNVLLLPEYSSKEKLKERLLKAITYAKGFGML",
# UBE3B_HUMAN
"MFTLSQTSRAWFIDRARQAREERLVQKERERAAVVIQAHVRSFLCRSRLQRDIRREIDDFFKADDPESTKRSALCIFKIARKLLFLFRIKEDNERFEKLCRSILSSMDAENEPKVWYVSLACSKDLTLLWIQQIKNILWYCCDFLKQLKPEILQDSRLITLYLTMLVTFTDTSTWKILRGKGESLRPAMNHICANIMGHLNQHGFYSVLQILLTRGLARPRPCLSKGTLTAAFSLALRPVIAAQFSDNLIRPFLIHIMSVPALVTHLSTVTPERLTVLESHDMLRKFIIFLRDQDRCRDVCESLEGCHTLCLMGNLLHLGSLSPRVLEEETDGFVSLLTQTLCYCRKYVSQKKSNLTHWHPVLGWFSQSVDYGLNESMHLITKQLQFLWGVPLIRIFFCDILSKKLLESQEPAHAQPASPQNVLPVKSLLKRAFQKSASVRNILRPVGGKRVDSAEVQKVCNICVLYQTSLTTLTQIRLQILTGLTYLDDLLPKLWAFICELGPHGGLKLFLECLNNDTEESKQLLAMLMLFCDCSRHLITILDDIEVYEEQISFKLEELVTISSFLNSFVFKMIWDGIVENAKGETLELFQSVHGWLMVLYERDCRRRFTPEDHWLRKDLKPSVLFQELDRDRKRAQLILQYIPHVIPHKNRVLLFRTMVTKEKEKLGLVETSSASPHVTHITIRRSRMLEDGYEQLRQLSQHAMKGVIRVKFVNDLGVDEAGIDQDGVFKEFLEEIIKRVFDPALNLFKTTSGDERLYPSPTSYIHENYLQLFEFVGKMLGKAVYEGIVVDVPFASFFLSQLLGHHHSVFYSSVDELPSLDSEFYKNLTSIKRYDGDITDLGLTLSYDEDVMGQLVCHELIPGGKTIPVTNENKISYIHLMAHFRMHTQIKNQTAALISGFRSIIKPEWIRMFSTPELQRLISGDNAEIDLEDLKKHTVYYGGFHGSHRVIIWLWDILASDFTPDERAMFLKFVTSCSRPPLLGFAYLKPPFSIRCVEVSDDQDTGDTLGSVLRGFFTIRKREPGGRLPTSSTCFNLLKLPNYSKKSVLREKLRYAISMNTGFELS",
# UBE3C_HUMAN
"MFSFEGDFKTRPKVSLGGASRKEEKASLLHRTQEERRKREEERRRLKNAIIIQSFIRGYRDRKQQYSIQRSAFDRCATLSQSGGAFPIANGPNLTLLVRQLLFFYKQNEDSKRLIWLYQNLIKHSSLFVKQLDGSERLTCLFQIKRLMSLCCRLLQNCNDDSLNVALPMRMLEVFSSENTYLPVLQDASYVVSVIEQILHYMIHNGYYRSLYLLINSKLPSSIEYSDLSRVPIAKILLENVLKPLHFTYNSCPEGARQQVFTAFTEEFLAAPFTDQIFHFIIPALADAQTVFPYEPFLNALLLIESRCSRKSGGAPWLFYFVLTVGENYLGALSEEGLLVYLRVLQTFLSQLPVSPASASCHDSASDSEEESEEADKPSSPEDGRLSVSYITEECLKKLDTKQQTNTLLNLVWRDSASEEVFTTMASVCHTLMVQHRMMVPKVRLLYSLAFNARFLRHLWFLISSMSTRMITGSMVPLLQVISRGSPMSFEDSSRIIPLFYLFSSLFSHSLISIHDNEFFGDPIEVVGQRQSSMMPFTLEELIMLSRCLRDACLGIIKLAYPETKPEVREEYITAFQSIGVTTSSEMQQCIQMEQKRWIQLFKVITNLVKMLKSRDTRRNFCPPNHWLSEQEDIKADKVTQLYVPASRHVWRFRRMGRIGPLQSTLDVGLESPPLSVSEERQLAVLTELPFVVPFEERVKIFQRLIYADKQEVQGDGPFLDGINVTIRRNYIYEDAYDKLSPENEPDLKKRIRVHLLNAHGLDEAGIDGGGIFREFLNELLKSGFNPNQGFFKTTNEGLLYPNPAAQMLVGDSFARHYYFLGRMLGKALYENMLVELPFAGFFLSKLLGTSADVDIHHLASLDPEVYKNLLFLKSYEDDVEELGLNFTVVNNDLGEAQVVELKFGGKDIPVTSANRIAYIHLVADYRLNRQIRQHCLAFRQGLANVVSLEWLRMFDQQEIQVLISGAQVPISLEDLKSFTNYSGGYSADHPVIKVFWRVVEGFTDEEKRKLLKFVTSCSRPPLLGFKELYPAFCIHNGGSDLERLPTASTCMNLLKLPEFYDETLLRSKLLYAIECAAGFELS",
# UBE3D_HUMAN
"MAASAAETRVFLEVRGQLQSALLILGEPKEGGMPMNISIMPSSLQMKTPEGCTEIQLPAEVRLVPSSCRGLQFVVGDGLHLRLQTQAKLGTKLISMFNQSSQTQECCTFYCQSCGEVIIKDRKLLRVLPLPSENWGALVGEWCCHPDPFANKSLHPQENDCFIGDSFFLVNLRTSLWQQRPELSPVEMCCVSSDNHCKLEPKANTKVICKRCKVMLGETVSSETTKFYMTEIIIQSSERSFPIIPRSWFVQSVIAQCLVQLSSARSTFRFTIQGQDDKVYILLWLLNSDSLVIESLRNSKYIKKFPLLENTFKADSSSAWSAVKVLYQPCIKSRNEKLVSLWESDISVHPLTLPSATCLELLLILSKSNANLPSSLRRVNSFQVAFLKM",
# UBE4A_HUMAN
"MTDQENNNNISSNPFAALFGSLADAKQFAAIQKEQLKQQSDELPASPDDSDNSVSESLDEFDYSVAEISRSFRSQQEICEQLNINHMIQRIFLITLDNSDPSLKSGNGIPSRCVYLEEMAVELEDQDWLDMSNVEQALFARLLLQDPGNHLINMTSSTTLNLSADRDAGERHIFCYLYSCFQRAKEEITKVPENLLPFAVQCRNLTVSNTRTVLLTPEIYVDQNIHEQLVDLMLEAIQGAHFEDVTEFLEEVIEALILDEEVRTFPEVMIPVFDILLGRIKDLELCQILLYAYLDILLYFTRQKDMAKVFVEYIQPKDPTNGQMYQKTLLGVILSISCLLKTPGVVENHGYFLNPSRSSPQEIKVQEANIHQFMAQFHEKIYQMLKNLLQLSPETKHCILSWLGNCLHANAGRTKIWANQMPEIFFQMYASDAFFLNLGAALLKLCQPFCKPRSSRLLTFNPTYCALKELNDEERKIKNVHMRGLDKETCLIPAVQEPKFPQNYNLVTENLALTEYTLYLGFHRLHDQMVKINQNLHRLQVAWRDAQQSSSPAADNLREQFERLMTIYLSTKTAMTEPQMLQNCLNLQVSMAVLLVQLAIGNEGSQPIELTFPLPDGYSSLAYVPEFFADNLGDFLIFLRRFADDILETSADSLEHVLHFITIFTGSIERMKNPHLRAKLAEVLEAVMPHLDQTPNPLVSSVFHRKRVFCNFQYAPQLAEALIKVFVDIEFTGDPHQFEQKFNYRRPMYPILRYMWGTDTYRESIKDLADYASKNLEAMNPPLFLRFLNLLMNDAIFLLDEAIQYLSKIKIQQIEKDRGEWDSLTPEARREKEAGLQMFGQLARFHNIMSNETIGTLAFLTSEIKSLFVHPFLAERIISMLNYFLQHLVGPKMGALKVKDFSEFDFKPQQLVSDICTIYLNLGDEENFCATVPKDGRSYSPTLFAQTVRVLKKINKPGNMIMAFSNLAERIKSLADLQQQEEETYADACDEFLDPIMSTLMCDPVVLPSSRVTVDRSTIARHLLSDQTDPFNRSPLTMDQIRPNTELKEKIQRWLAERKQQKEQLE",
# UBE4B_HUMAN
"MEELSADEIRRRRLARLAGGQTSQPTTPLTSPQRENPPGPPIAASAPGPSQSLGLNVHNMTPATSPIGASGVAHRSQSSEGVSSLSSSPSNSLETQSQSLSRSQSMDIDGVSCEKSMSQVDVDSGIENMEVDENDRREKRSLSDKEPSSGPEVSEEQALQLVCKIFRVSWKDRDRDVIFLSSLSAQFKQNPKEVFSDFKDLIGQILMEVLMMSTQTRDENPFASLTATSQPIAAAARSPDRNLLLNTGSNPGTSPMFCSVASFGASSLSSLYESSPAPTPSFWSSVPVMGPSLASPSRAASQLAVPSTPLSPHSAASGTAAGSQPSSPRYRPYTVTHPWASSGVSILSSSPSPPALASSPQAVPASSSRQRPSSTGPPLPPASPSATSRRPSSLRISPSLGASGGASNWDSYSDHFTIETCKETDMLNYLIECFDRVGIEEKKAPKMCSQPAVSQLLSNIRSQCISHTALVLQGSLTQPRSLQQPSFLVPYMLCRNLPYGFIQELVRTTHQDEEVFKQIFIPILQGLALAAKECSLDSDYFKYPLMALGELCETKFGKTHPVCNLVASLRLWLPKSLSPGCGRELQRLSYLGAFFSFSVFAEDDVKVVEKYFSGPAITLENTRVVSQSLQHYLELGRQELFKILHSILLNGETREAALSYMAAVVNANMKKAQMQTDDRLVSTDGFMLNFLWVLQQLSTKIKLETVDPTYIFHPRCRITLPNDETRVNATMEDVNDWLTELYGDQPPFSEPKFPTECFFLTLHAHHLSILPSCRRYIRRLRAIRELNRTVEDLKNNESQWKDSPLATRHREMLKRCKTQLKKLVRCKACADAGLLDESFLRRCLNFYGLLIQLLLRILDPAYPDITLPLNSDVPKVFAALPEFYVEDVAEFLFFIVQYSPQALYEPCTQDIVMFLVVMLCNQNYIRNPYLVAKLVEVMFMTNPAVQPRTQKFFEMIENHPLSTKLLVPSLMKFYTDVEHTGATSEFYDKFTIRYHISTIFKSLWQNIAHHGTFMEEFNSGKQFVRYINMLINDTTFLLDESLESLKRIHEVQEEMKNKEQWDQLPRDQQQARQSQLAQDERVSRSYLALATETVDMFHILTKQVQKPFLRPELGPRLAAMLNFNLQQLCGPKCRDLKVENPEKYGFEPKKLLDQLTDIYLQLDCARFAKAIADDQRSYSKELFEEVISKMRKAGIKSTIAIEKFKLLAEKVEEIVAKNARAEIDYSDAPDEFRDPLMDTLMTDPVRLPSGTIMDRSIILRHLLNSPTDPFNRQTLTESMLEPVPELKEQIQAWMREKQNSDH",
# RNF37_HUMAN
"MVINLCLPQFRPRIHCNKISADGYEVENLISEDLTKRSHGFRTEYFIKPPVYVTVSFPFNVEICRINIDLTAGGGQNVTGLEMYTSASSSRVSWNTPQCRTLGPAEPSVPDKEAFTLVGKVLLKNQSQVVFSHRGFKARPPFGAMEATLPSPAVVAQELWNKGALSLSHVAHLRICITHVTGGGIPCIKRLEVWGQPAKTCSQEVIDSILLVTSENLPQDVALQAPALPMESDCDPGDQPESQQAPSSLQKLAEIIQDVPEEFLDPITLEIMPCPMLLPSGKVIDQSTLEKCNRSEATWGRVPSDPFTGVAFTPHSQPLPHPSLKARIDHFLLQHSIPGCHLLGRAQTALAVIPSSIVLPSQKRKIEQAEHVPDSNFGVNASCFSATSPLVLPTTSEHTAKKMKATNEPSLTHMDCSTGPLSHEQKLSQSLEIALASTLGSMPSFTARLTRGQLQHLGTRGSNTSWRPGTGSEQPGSILGPECASCKRVFSPYFKKEPVYQLPCGHLLCRPCLGEKQRSLPMTCTACQRPVASQDVLRVHF",
# UBR1_HUMAN
"MADEEAGGTERMEISAELPQTPQRLASWWDQQVDFYTAFLHHLAQLVPEIYFAEMDPDLEKQEESVQMSIFTPLEWYLFGEDPDICLEKLKHSGAFQLCGRVFKSGETTYSCRDCAIDPTCVLCMDCFQDSVHKNHRYKMHTSTGGGFCDCGDTEAWKTGPFCVNHEPGRAGTIKENSRCPLNEEVIVQARKIFPSVIKYVVEMTIWEEEKELPPELQIREKNERYYCVLFNDEHHSYDHVIYSLQRALDCELAEAQLHTTAIDKEGRRAVKAGAYAACQEAKEDIKSHSENVSQHPLHVEVLHSEIMAHQKFALRLGSWMNKIMSYSSDFRQIFCQACLREEPDSENPCLISRLMLWDAKLYKGARKILHELIFSSFFMEMEYKKLFAMEFVKYYKQLQKEYISDDHDRSISITALSVQMFTVPTLARHLIEEQNVISVITETLLEVLPEYLDRNNKFNFQGYSQDKLGRVYAVICDLKYILISKPTIWTERLRMQFLEGFRSFLKILTCMQGMEEIRRQVGQHIEVDPDWEAAIAIQMQLKNILLMFQEWCACDEELLLVAYKECHKAVMRCSTSFISSSKTVVQSCGHSLETKSYRVSEDLVSIHLPLSRTLAGLHVRLSRLGAVSRLHEFVSFEDFQVEVLVEYPLRCLVLVAQVVAEMWRRNGLSLISQVFYYQDVKCREEMYDKDIIMLQIGASLMDPNKFLLLVLQRYELAEAFNKTISTKDQDLIKQYNTLIEEMLQVLIYIVGERYVPGVGNVTKEEVTMREIIHLLCIEPMPHSAIAKNLPENENNETGLENVINKVATFKKPGVSGHGVYELKDESLKDFNMYFYHYSKTQHSKAEHMQKKRRKQENKDEALPPPPPPEFCPAFSKVINLLNCDIMMYILRTVFERAIDTDSNLWTEGMLQMAFHILALGLLEEKQQLQKAPEEEVTFDFYHKASRLGSSAMNIQMLLEKLKGIPQLEGQKDMITWILQMFDTVKRLREKSCLIVATTSGSESIKNDEITHDKEKAERKRKAEAARLHRQKIMAQMSALQKNFIETHKLMYDNTSEMPGKEDSIMEEESTPAVSDYSRIALGPKRGPSVTEKEVLTCILCQEEQEVKIENNAMVLSACVQKSTALTQHRGKPIELSGEALDPLFMDPDLAYGTYTGSCGHVMHAVCWQKYFEAVQLSSQQRIHVDLFDLESGEYLCPLCKSLCNTVIPIIPLQPQKINSENADALAQLLTLARWIQTVLARISGYNIRHAKGENPIPIFFNQGMGDSTLEFHSILSFGVESSIKYSNSIKEMVILFATTIYRIGLKVPPDERDPRVPMLTWSTCAFTIQAIENLLGDEGKPLFGALQNRQHNGLKALMQFAVAQRITCPQVLIQKHLVRLLSVVLPNIKSEDTPCLLSIDLFHVLVGAVLAFPSLYWDDPVDLQPSSVSSSYNHLYLFHLITMAHMLQILLTVDTGLPLAQVQEDSEEAHSASSFFAEISQYTSGSIGCDIPGWYLWVSLKNGITPYLRCAALFFHYLLGVTPPEELHTNSAEGEYSALCSYLSLPTNLFLLFQEYWDTVRPLLQRWCADPALLNCLKQKNTVVRYPRKRNSLIELPDDYSCLLNQASHFRCPRSADDERKHPVLCLFCGAILCSQNICCQEIVNGEEVGACIFHALHCGAGVCIFLKIRECRVVLVEGKARGCAYPAPYLDEYGETDPGLKRGNPLHLSRERYRKLHLVWQQHCIIEEIARSQETNQMLFGFNWQLL",
# UBR2_HUMAN
"MASELEPEVQAIDRSLLECSAEEIAGKWLQATDLTREVYQHLAHYVPKIYCRGPNPFPQKEDMLAQHVLLGPMEWYLCGEDPAFGFPKLEQANKPSHLCGRVFKVGEPTYSCRDCAVDPTCVLCMECFLGSIHRDHRYRMTTSGGGGFCDCGDTEAWKEGPYCQKHELNTSEIEEEEDPLVHLSEDVIARTYNIFAITFRYAVEILTWEKESELPADLEMVEKSDTYYCMLFNDEVHTYEQVIYTLQKAVNCTQKEAIGFATTVDRDGRRSVRYGDFQYCEQAKSVIVRNTSRQTKPLKVQVMHSSIVAHQNFGLKLLSWLGSIIGYSDGLRRILCQVGLQEGPDGENSSLVDRLMLSDSKLWKGARSVYHQLFMSSLLMDLKYKKLFAVRFAKNYQQLQRDFMEDDHERAVSVTALSVQFFTAPTLARMLITEENLMSIIIKTFMDHLRHRDAQGRFQFERYTALQAFKFRRVQSLILDLKYVLISKPTEWSDELRQKFLEGFDAFLELLKCMQGMDPITRQVGQHIEMEPEWEAAFTLQMKLTHVISMMQDWCASDEKVLIEAYKKCLAVLMQCHGGYTDGEQPITLSICGHSVETIRYCVSQEKVSIHLPVSRLLAGLHVLLSKSEVAYKFPELLPLSELSPPMLIEHPLRCLVLCAQVHAGMWRRNGFSLVNQIYYYHNVKCRREMFDKDVVMLQTGVSMMDPNHFLMIMLSRFELYQIFSTPDYGKRFSSEITHKDVVQQNNTLIEEMLYLIIMLVGERFSPGVGQVNATDEIKREIIHQLSIKPMAHSELVKSLPEDENKETGMESVIEAVAHFKKPGLTGRGMYELKPECAKEFNLYFYHFSRAEQSKAEEAQRKLKRQNREDTALPPPVLPPFCPLFASLVNILQSDVMLCIMGTILQWAVEHNGYAWSESMLQRVLHLIGMALQEEKQHLENVTEEHVVTFTFTQKISKPGEAPKNSPSILAMLETLQNAPYLEVHKDMIRWILKTFNAVKKMRESSPTSPVAETEGTIMEESSRDKDKAERKRKAEIARLRREKIMAQMSEMQRHFIDENKELFQQTLELDASTSAVLDHSPVASDMTLTALGPAQTQVPEQRQFVTCILCQEEQEVKVESRAMVLAAFVQRSTVLSKNRSKFIQDPEKYDPLFMHPDLSCGTHTSSCGHIMHAHCWQRYFDSVQAKEQRRQQRLRLHTSYDVENGEFLCPLCECLSNTVIPLLLPPRNIFNNRLNFSDQPNLTQWIRTISQQIKALQFLRKEESTPNNASTKNSENVDELQLPEGFRPDFRPKIPYSESIKEMLTTFGTATYKVGLKVHPNEEDPRVPIMCWGSCAYTIQSIERILSDEDKPLFGPLPCRLDDCLRSLTRFAAAHWTVASVSVVQGHFCKLFASLVPNDSHEELPCILDIDMFHLLVGLVLAFPALQCQDFSGISLGTGDLHIFHLVTMAHIIQILLTSCTEENGMDQENPPCEEESAVLALYKTLHQYTGSALKEIPSGWHLWRSVRAGIMPFLKCSALFFHYLNGVPSPPDIQVPGTSHFEHLCSYLSLPNNLICLFQENSEIMNSLIESWCRNSEVKRYLEGERDAIRYPRESNKLINLPEDYSSLINQASNFSCPKSGGDKSRAPTLCLVCGSLLCSQSYCCQTELEGEDVGACTAHTYSCGSGVGIFLRVRECQVLFLAGKTKGCFYSPPYLDDYGETDQGLRRGNPLHLCKERFKKIQKLWHQHSVTEEIGHAQEANQTLVGIDWQHL",
# UBR3_HUMAN
"MAAAAAAAVGGQQPSQPELPAPGLALDKAATAAHLKAALSRPDNRAGAEELQALLERVLSAERPLAAAAGGEDAAAAGGGGGPGAAEEEALEWCKCLLAGGGGYDEFCAAVRAYDPAALCGLVWTANFVAYRCRTCGISPCMSLCAECFHQGDHTGHDFNMFRSQAGGACDCGDSNVMRESGFCKRHQIKSSSNIPCVPKDLLMMSEFVLPRFIFCLIQYLREGYNEPAADGPSEKDLNKVLQLLEPQISFLEDLTKMGGAMRSVLTQVLTNQQNYKDLTSGLGENACVKKSHEKYLIALKSSGLTYPEDKLVYGVQEPSAGTSSLAVQGFIGATGTLGQVDSSDEDDQDGSQGLGKRKRVKLSSGTKDQSIMDVLKHKSFLEELLFWTIKYEFPQKMVTFLLNMLPDQEYKVAFTKTFVQHYAFIMKTLKKSHESDTMSNRIVHISVQLFSNEELARQVTEECQLLDIMVTVLLYMMESCLIKSELQDEENSLHVVVNCGEALLKNNTYWPLVSDFINILSHQSVAKRFLEDHGLLVTWMNFVSFFQGMNLNKRELNEHVEFESQTYYAAFAAELEACAQPMWGLLSHCKVRETQEYTRNVVRYCLEALQDWFDAINFVDEPAPNQVTFHLPLHRYYAMFLSKAVKCQELDLDSVLPDQEMLMKLMIHPLQIQASLAEIHSNMWVRNGLQIKGQAMTYVQSHFCNSMIDPDIYLLQVCASRLDPDYFISSVFERFKVVDLLTMASQHQNTVLDAEHERSMLEGALTFLVILLSLRLHLGMSDDEILRAEMVAQLCMNDRTHSSLLDLIPENPNPKSGIIPGSYSFESVLSAVADFKAPVFEPGGSMQQGMYTPKAEVWDQEFDPVMVILRTVYRRDVQSAMDRYTAFLKQSGKFPGNPWPPYKKRTSLHPSYKGLMRLLHCKTLHIVLFTLLYKILMDHQNLSEHVLCMVLYLIELGLENSAEEESDEEASVGGPERCHDSWFPGSNLVSNMRHFINYVRVRVPETAPEVKRDSPASTSSDNLGSLQNSGTAQVFSLVAERRKKFQEIINRSSSEANQVVRPKTSSKWSAPGSAPQLTTAILEIKESILSLLIKLHHKLSGKQNSYYPPWLDDIEILIQPEIPKYSHGDGITAVERILLKAASQSRMNKRIIEEICRKVTPPVPPKKVTAAEKKTLDKEERRQKARERQQKLLAEFASRQKSFMETAMDVDSPENDIPMEITTAEPQVSEAVYDCVICGQSGPSSEDRPTGLVVLLQASSVLGQCRDNVEPKKLPISEEEQIYPWDTCAAVHDVRLSLLQRYFKDSSCLLAVSIGWEGGVYVQTCGHTLHIDCHKSYMESLRNDQVLQGFSVDKGEFTCPLCRQFANSVLPCYPGSNVENNPWQRPSNKSIQDLIKEVEELQGRPGAFPSETNLSKEMESVMKDIKNTTQKKYRDYSKTPGSPDNDFLFMYSVARTNLELELIHRGGNLCSGGASTAGKRSCLNQLFHVLALHMRLYSIDSEYNPWRKLTQLEEMNPQLGYEEQQPEVPILYHDVTSLLLIQILMMPQPLRKDHFTCIVKVLFTLLYTQALAALSVKCSEEDRSAWKHAGALKKSTCDAEKSYEVLLSFVISELFKGKLYHEEGTQECAMVNPIAWSPESMEKCLQDFCLPFLRITSLLQHHLFGEDLPSCQEEEEFSVLASCLGLLPTFYQTEHPFISASCLDWPVPAFDIITQWCFEIKSFTERHAEQGKALLIQESKWKLPHLLQLPENYNTIFQYYHRKTCSVCTKVPKDPAVCLVCGTFVCLKGLCCKQQSYCECVLHSQNCGAGTGIFLLINASVIIIIRGHRFCLWGSVYLDAHGEEDRDLRRGKPLYICKERYKVLEQQWISHTFDHINKRWGPHYNGL",
# UBR4_HUMAN
"MATSGGEEAAAAAPAPGTPATGADTTPGWEVAVRPLLSASYSAFEMKELPQLVASVIESESEILHHEKQYEPFYSSFVALSTHYITTVCSLIPRNQLQSVAAACKVLIEFSLLRLENPDEACAVSQKHLILLIKGLCTGCSRLDRTEIITFTAMMKSAKLPQTVKTLSDVEDQKELASPVSPELRQKEVQMNFLNQLTSVFNPRTVASQPISTQTLVEGENDEQSSTDQASAIKTKNVFIAQNVASLQELGGSEKLLRVCLNLPYFLRYINRFQDAVLANSFFIMPATVADATAVRNGFHSLVIDVTMALDTLSLPVLEPLNPSRLQDVTVLSLSCLYAGVSVATCMAILHVGSAQQVRTGSTSSKEDDYESDAATIVQKCLEIYDMIGQAISSSRRAGGEHYQNFQLLGAWCLLNSLFLILNLSPTALADKGKEKDPLAALRVRDILSRTKEGVGSPKLGPGKGHQGFGVLSVILANHAIKLLTSLFQDLQVEALHKGWETDGPPAALSIMAQSTSIQRIQRLIDSVPLMNLLLTLLSTSYRKACVLQRQRKGSMSSDASASTDSNTYYEDDFSSTEEDSSQDDDSEPILGQWFEETISPSKEKAAPPPPPPPPPLESSPRVKSPSKQAPGEKGNILASRKDPELFLGLASNILNFITSSMLNSRNNFIRNYLSVSLSEHHMATLASIIKEVDKDGLKGSSDEEFAAALYHFNHSLVTSDLQSPNLQNTLLQQLGVAPFSEGPWPLYIHPQSLSVLSRLLLIWQHKASAQGDPDVPECLKVWDRFLSTMKQNALQGVVPSETEDLNVEHLQMLLLIFHNFTETGRRAILSLFVQIIQELSVNMDAQMRFVPLILARLLLIFDYLLHQYSKAPVYLFEQVQHNLLSPPFGWASGSQDSNSRRATTPLYHGFKEVEENWSKHFSSDAVPHPRFYCVLSPEASEDDLNRLDSVACDVLFSKLVKYDELYAALTALLAAGSQLDTVRRKENKNVTALEACALQYYFLILWRILGILPPSKTYINQLSMNSPEMSECDILHTLRWSSRLRISSYVNWIKDHLIKQGMKAEHASSLLELASTTKCSSVKYDVEIVEEYFARQISSFCSIDCTTILQLHEIPSLQSIYTLDAAISKVQVSLDEHFSKMAAETDPHKSSEITKNLLPATLQLIDTYASFTRAYLLQNFNEEGTTEKPSKEKLQGFAAVLAIGSSRCKANTLGPTLVQNLPSSVQTVCESWNNINTNEFPNIGSWRNAFANDTIPSESYISAVQAAHLGTLCSQSLPLAASLKHTLLSLVRLTGDLIVWSDEMNPPQVIRTLLPLLLESSTESVAEISSNSLERILGPAESDEFLARVYEKLITGCYNILANHADPNSGLDESILEECLQYLEKQLESSQARKAMEEFFSDSGELVQIMMATANENLSAKFCNRVLKFFTKLFQLTEKSPNPSLLHLCGSLAQLACVEPVRLQAWLTRMTTSPPKDSDQLDVIQENRQLLQLLTTYIVRENSQVGEGVCAVLLGTLTPMATEMLANGDGTGFPELMVVMATLASAGQGAGHLQLHNAAVDWLSRCKKYLSQKNVVEKLNANVMHGKHVMILECTCHIMSYLADVTNALSQSNGQGPSHLSVDGEERAIEVDSDWVEELAVEEEDSQAEDSDEDSLCNKLCTFTITQKEFMNQHWYHCHTCKMVDGVGVCTVCAKVCHKDHEISYAKYGSFFCDCGAKEDGSCLALVKRTPSSGMSSTMKESAFQSEPRISESLVRHASTSSPADKAKVTISDGKVADEEKPKKSSLCRTVEGCREELQNQANFSFAPLVLDMLNFLMDAIQTNFQQASAVGSSSRAQQALSELHTVEKAVEMTDQLMVPTLGSQEGAFENVRMNYSGDQGQTIRQLISAHVLRRVAMCVLSSPHGRRQHLAVSHEKGKITVLQLSALLKQADSSKRKLTLTRLASAPVPFTVLSLTGNPCKEDYLAVCGLKDCHVLTFSSSGSVSDHLVLHPQLATGNFIIKAVWLPGSQTELAIVTADFVKIYDLCVDALSPTFYFLLPSSKIRDVTFLFNEEGKNIIVIMSSAGYIYTQLMEEASSAQQGPFYVTNVLEINHEDLKDSNSQVAGGGVSVYYSHVLQMLFFSYCQGKSFAATISRTTLEVLQLFPINIKSSNGGSKTSPALCQWSEVMNHPGLVCCVQQTTGVPLVVMVKPDTFLIQEIKTLPAKAKIQDMVAIRHTACNEQQRTTMILLCEDGSLRIYMANVENTSYWLQPSLQPSSVISIMKPVRKRKTATITTRTSSQVTFPIDFFEHNQQLTDVEFGGNDLLQVYNAQQIKHRLNSTGMYVANTKPGGFTIEISNNNSTMVMTGMRIQIGTQAIERAPSYIEIFGRTMQLNLSRSRWFDFPFTREEALQADKKLNLFIGASVDPAGVTMIDAVKIYGKTKEQFGWPDEPPEEFPSASVSNICPSNLNQSNGTGDSDSAAPTTTSGTVLERLVVSSLEALESCFAVGPIIEKERNKNAAQELATLLLSLPAPASVQQQSKSLLASLHTSRSAYHSHKDQALLSKAVQCLNTSSKEGKDLDPEVFQRLVITARSIAIMRPNNLVHFTESKLPQMETEGMDEGKEPQKQLEGDCCSFITQLVNHFWKLHASKPKNAFLAPACLPGLTHIEATVNALVDIIHGYCTCELDCINTASKIYMQMLLCPDPAVSFSCKQALIRVLRPRNKRRHVTLPSSPRSNTPMGDKDDDDDDDADEKMQSSGIPNGGHIRQESQEQSEVDHGDFEMVSESMVLETAENVNNGNPSPLEALLAGAEGFPPMLDIPPDADDETMVELAIALSLQQDQQGSSSSALGLQSLGLSGQAPSSSSLDAGTLSDTTASAPASDDEGSTAATDGSTLRTSPADHGGSVGSESGGSAVDSVAGEHSVSGRSSAYGDATAEGHPAGPGSVSSSTGAISTTTGHQEGDGSEGEGEGETEGDVHTSNRLHMVRLMLLERLLQTLPQLRNVGGVRAIPYMQVILMLTTDLDGEDEKDKGALDNLLSQLIAELGMDKKDVSKKNERSALNEVHLVVMRLLSVFMSRTKSGSKSSICESSSLISSATAAALLSSGAVDYCLHVLKSLLEYWKSQQNDEEPVATSQLLKPHTTSSPPDMSPFFLRQYVKGHAADVFEAYTQLLTEMVLRLPYQIKKITDTNSRIPPPVFDHSWFYFLSEYLMIQQTPFVRRQVRKLLLFICGSKEKYRQLRDLHTLDSHVRGIKKLLEEQGIFLRASVVTASSGSALQYDTLISLMEHLKACAEIAAQRTINWQKFCIKDDSVLYFLLQVSFLVDEGVSPVLLQLLSCALCGSKVLAALAASSGSSSASSSSAPVAASSGQATTQSKSSTKKSKKEEKEKEKDGETSGSQEDQLCTALVNQLNKFADKETLIQFLRCFLLESNSSSVRWQAHCLTLHIYRNSSKSQQELLLDLMWSIWPELPAYGRKAAQFVDLLGYFSLKTPQTEKKLKEYSQKAVEILRTQNHILTNHPNSNIYNTLSGLVEFDGYYLESDPCLVCNNPEVPFCYIKLSSIKVDTRYTTTQQVVKLIGSHTISKVTVKIGDLKRTKMVRTINLYYNNRTVQAIVELKNKPARWHKAKKVQLTPGQTEVKIDLPLPIVASNLMIEFADFYENYQASTETLQCPRCSASVPANPGVCGNCGENVYQCHKCRSINYDEKDPFLCNACGFCKYARFDFMLYAKPCCAVDPIENEEDRKKAVSNINTLLDKADRVYHQLMGHRPQLENLLCKVNEAAPEKPQDDSGTAGGISSTSASVNRYILQLAQEYCGDCKNSFDELSKIIQKVFASRKELLEYDLQQREAATKSSRTSVQPTFTASQYRALSVLGCGHTSSTKCYGCASAVTEHCITLLRALATNPALRHILVSQGLIRELFDYNLRRGAAAMREEVRQLMCLLTRDNPEATQQMNDLIIGKVSTALKGHWANPDLASSLQYEMLLLTDSISKEDSCWELRLRCALSLFLMAVNIKTPVVVENITLMCLRILQKLIKPPAPTSKKNKDVPVEALTTVKPYCNEIHAQAQLWLKRDPKASYDAWKKCLPIRGIDGNGKAPSKSELRHLYLTEKYVWRWKQFLSRRGKRTSPLDLKLGHNNWLRQVLFTPATQAARQAACTIVEALATIPSRKQQVLDLLTSYLDELSIAGECAAEYLALYQKLITSAHWKVYLAARGVLPYVGNLITKEIARLLALEEATLSTDLQQGYALKSLTGLLSSFVEVESIKRHFKSRLVGTVLNGYLCLRKLVVQRTKLIDETQDMLLEMLEDMTTGTESETKAFMAVCIETAKRYNLDDYRTPVFIFERLCSIIYPEENEVTEFFVTLEKDPQQEDFLQGRMPGNPYSSNEPGIGPLMRDIKNKICQDCDLVALLEDDSGMELLVNNKIISLDLPVAEVYKKVWCTTNEGEPMRIVYRMRGLLGDATEEFIESLDSTTDEEEDEEEVYKMAGVMAQCGGLECMLNRLAGIRDFKQGRHLLTVLLKLFSYCVKVKVNRQQLVKLEMNTLNVMLGTLNLALVAEQESKDSGGAAVAEQVLSIMEIILDESNAEPLSEDKGNLLLTGDKDQLVMLLDQINSTFVRSNPSVLQGLLRIIPYLSFGEVEKMQILVERFKPYCNFDKYDEDHSGDDKVFLDCFCKIAAGIKNNSNGHQLKDLILQKGITQNALDYMKKHIPSAKNLDADIWKKFLSRPALPFILRLLRGLAIQHPGTQVLIGTDSIPNLHKLEQVSSDEGIGTLAENLLEALREHPDVNKKIDAARRETRAEKKRMAMAMRQKALGTLGMTTNEKGQVVTKTALLKQMEELIEEPGLTCCICREGYKFQPTKVLGIYTFTKRVALEEMENKPRKQQGYSTVSHFNIVHYDCHLAAVRLARGREEWESAALQNANTKCNGLLPVWGPHVPESAFATCLARHNTYLQECTGQREPTYQLNIHDIKLLFLRFAMEQSFSADTGGGGRESNIHLIPYIIHTVLYVLNTTRATSREEKNLQGFLEQPKEKWVESAFEVDGPYYFTVLALHILPPEQWRATRVEILRRLLVTSQARAVAPGGATRLTDKAVKDYSAYRSSLLFWALVDLIYNMFKKVPTSNTEGGWSCSLAEYIRHNDMPIYEAADKALKTFQEEFMPVETFSEFLDVAGLLSEITDPESFLKDLLNSVP",
# UBR5_HUMAN
"MTSIHFVVHPLPGTEDQLNDRLREVSEKLNKYNLNSHPPLNVLEQATIKQCVVGPNHAAFLLEDGRVCRIGFSVQPDRLELGKPDNNDGSKLNSNSGAGRTSRPGRTSDSPWFLSGSETLGRLAGNTLGSRWSSGVGGSGGGSSGRSSAGARDSRRQTRVIRTGRDRGSGLLGSQPQPVIPASVIPEELISQAQVVLQGKSRSVIIRELQRTNLDVNLAVNNLLSRDDEDGDDGDDTASESYLPGEDLMSLLDADIHSAHPSVIIDADAMFSEDISYFGYPSFRRSSLSRLGSSRVLLLPLERDSELLRERESVLRLRERRWLDGASFDNERGSTSKEGEPNLDKKNTPVQSPVSLGEDLQWWPDKDGTKFICIGALYSELLAVSSKGELYQWKWSESEPYRNAQNPSLHHPRATFLGLTNEKIVLLSANSIRATVATENNKVATWVDETLSSVASKLEHTAQTYSELQGERIVSLHCCALYTCAQLENSLYWWGVVPFSQRKKMLEKARAKNKKPKSSAGISSMPNITVGTQVCLRNNPLYHAGAVAFSISAGIPKVGVLMESVWNMNDSCRFQLRSPESLKNMEKASKTTEAKPESKQEPVKTEMGPPPSPASTCSDASSIASSASMPYKRRRSTPAPKEEEKVNEEQWSLREVVFVEDVKNVPVGKVLKVDGAYVAVKFPGTSSNTNCQNSSGPDADPSSLLQDCRLLRIDELQVVKTGGTPKVPDCFQRTPKKLCIPEKTEILAVNVDSKGVHAVLKTGNWVRYCIFDLATGKAEQENNFPTSSIAFLGQNERNVAIFTAGQESPIILRDGNGTIYPMAKDCMGGIRDPDWLDLPPISSLGMGVHSLINLPANSTIKKKAAVIIMAVEKQTLMQHILRCDYEACRQYLMNLEQAVVLEQNLQMLQTFISHRCDGNRNILHACVSVCFPTSNKETKEEEEAERSERNTFAERLSAVEAIANAISVVSSNGPGNRAGSSSSRSLRLREMMRRSLRAAGLGRHEAGASSSDHQDPVSPPIAPPSWVPDPPAMDPDGDIDFILAPAVGSLTTAATGTGQGPSTSTIPGPSTEPSVVESKDRKANAHFILKLLCDSVVLQPYLRELLSAKDARGMTPFMSAVSGRAYPAAITILETAQKIAKAEISSSEKEEDVFMGMVCPSGTNPDDSPLYVLCCNDTCSFTWTGAEHINQDIFECRTCGLLESLCCCTECARVCHKGHDCKLKRTSPTAYCDCWEKCKCKTLIAGQKSARLDLLYRLLTATNLVTLPNSRGEHLLLFLVQTVARQTVEHCQYRPPRIREDRNRKTASPEDSDMPDHDLEPPRFAQLALERVLQDWNALKSMIMFGSQENKDPLSASSRIGHLLPEEQVYLNQQSGTIRLDCFTHCLIVKCTADILLLDTLLGTLVKELQNKYTPGRREEAIAVTMRFLRSVARVFVILSVEMASSKKKNNFIPQPIGKCKRVFQALLPYAVEELCNVAESLIVPVRMGIARPTAPFTLASTSIDAMQGSEELFSVEPLPPRPSSDQSSSSSQSQSSYIIRNPQQRRISQSQPVRGRDEEQDDIVSADVEEVEVVEGVAGEEDHHDEQEEHGEENAEAEGQHDEHDEDGSDMELDLLAAAETESDSESNHSNQDNASGRRSVVTAATAGSEAGASSVPAFFSEDDSQSNDSSDSDSSSSQSDDIEQETFMLDEPLERTTNSSHANGAAQAPRSMQWAVRNTQHQRAASTAPSSTSTPAASSAGLIYIDPSNLRRSGTISTSAAAAAAALEASNASSYLTSASSLARAYSIVIRQISDLMGLIPKYNHLVYSQIPAAVKLTYQDAVNLQNYVEEKLIPTWNWMVSIMDSTEAQLRYGSALASAGDPGHPNHPLHASQNSARRERMTAREEASLRTLEGRRRATLLSARQGMMSARGDFLNYALSLMRSHNDEHSDVLPVLDVCSLKHVAYVFQALIYWIKAMNQQTTLDTPQLERKRTRELLELGIDNEDSEHENDDDTNQSATLNDKDDDSLPAETGQNHPFFRRSDSMTFLGCIPPNPFEVPLAEAIPLADQPHLLQPNARKEDLFGRPSQGLYSSSASSGKCLMEVTVDRNCLEVLPTKMSYAANLKNVMNMQNRQKKEGEEQPVLPEETESSKPGPSAHDLAAQLKSSLLAEIGLTESEGPPLTSFRPQCSFMGMVISHDMLLGRWRLSLELFGRVFMEDVGAEPGSILTELGGFEVKESKFRREMEKLRNQQSRDLSLEVDRDRDLLIQQTMRQLNNHFGRRCATTPMAVHRVKVTFKDEPGEGSGVARSFYTAIAQAFLSNEKLPNLECIQNANKGTHTSLMQRLRNRGERDREREREREMRRSSGLRAGSRRDRDRDFRRQLSIDTRPFRPASEGNPSDDPEPLPAHRQALGERLYPRVQAMQPAFASKITGMLLELSPAQLLLLLASEDSLRARVDEAMELIIAHGRENGADSILDLGLVDSSEKVQQENRKRHGSSRSVVDMDLDDTDDGDDNAPLFYQPGKRGFYTPRPGKNTEARLNCFRNIGRILGLCLLQNELCPITLNRHVIKVLLGRKVNWHDFAFFDPVMYESLRQLILASQSSDADAVFSAMDLAFAIDLCKEEGGGQVELIPNGVNIPVTPQNVYEYVRKYAEHRMLVVAEQPLHAMRKGLLDVLPKNSLEDLTAEDFRLLVNGCGEVNVQMLISFTSFNDESGENAEKLLQFKRWFWSIVEKMSMTERQDLVYFWTSSPSLPASEEGFQPMPSITIRPPDDQHLPTANTCISRLYVPLYSSKQILKQKLLLAIKTKNFGFV",
# UBR7_HUMAN
"MAGAEGAAGRQSELEPVVSLVDVLEEDEELENEACAVLGGSDSEKCSYSQGSVKRQALYACSTCTPEGEEPAGICLACSYECHGSHKLFELYTKRNFRCDCGNSKFKNLECKLLPDKAKVNSGNKYNDNFFGLYCICKRPYPDPEDEIPDEMIQCVVCEDWFHGRHLGAIPPESGDFQEMVCQACMKRCSFLWAYAAQLAVTKISTEDDGLVRNIDGIGDQEVIKPENGEHQDSTLKEDVPEQGKDDVREVKVEQNSEPCAGSSSESDLQTVFKNESLNAESKSGCKLQELKAKQLIKKDTATYWPLNWRSKLCTCQDCMKMYGDLDVLFLTDEYDTVLAYENKGKIAQATDRSDPLMDTLSSMNRVQQVELICEYNDLKTELKDYLKRFADEGTVVKREDIQQFFEEFQSKKRRRVDGMQYYCS",
# UHRF1_HUMAN
"MWIQVRTMDGRQTHTVDSLSRLTKVEELRRKIQELFHVEPGLQRLFYRGKQMEDGHTLFDYEVRLNDTIQLLVRQSLVLPHSTKERDSELSDTDSGCCLGQSESDKSSTHGEAAAETDSRPADEDMWDETELGLYKVNEYVDARDTNMGAWFEAQVVRVTRKAPSRDEPCSSTSRPALEEDVIYHVKYDDYPENGVVQMNSRDVRARARTIIKWQDLEVGQVVMLNYNPDNPKERGFWYDAEISRKRETRTARELYANVVLGDDSLNDCRIIFVDEVFKIERPGEGSPMVDNPMRRKSGPSCKHCKDDVNRLCRVCACHLCGGRQDPDKQLMCDECDMAFHIYCLDPPLSSVPSEDEWYCPECRNDASEVVLAGERLRESKKKAKMASATSSSQRDWGKGMACVGRTKECTIVPSNHYGPIPGIPVGTMWRFRVQVSESGVHRPHVAGIHGRSNDGAYSLVLAGGYEDDVDHGNFFTYTGSGGRDLSGNKRTAEQSCDQKLTNTNRALALNCFAPINDQEGAEAKDWRSGKPVRVVRNVKGGKNSKYAPAEGNRYDGIYKVVKYWPEKGKSGFLVWRYLLRRDDDEPGPWTKEGKDRIKKLGLTMQYPEGYLEALANREREKENSKREEEEQQEGGFASPRTGKGKWKRKSAGGGPSRAGSPRRTSKKTKVEPYSLTAQQSSLIREDKSNAKLWNEVLASLKDRPASGSPFQLFLSKVEETFQCICCQELVFRPITTVCQHNVCKDCLDRSFRAQVFSCPACRYDLGRSYAMQVNQPLQTVLNQLFPGYGNGR",
# UHRF2_HUMAN
"MWIQVRTIDGSKTCTIEDVSRKATIEELRERVWALFDVRPECQRLFYRGKQLENGYTLFDYDVGLNDIIQLLVRPDPDHLPGTSTQIEAKPCSNSPPKVKKAPRVGPSNQPSTSARARLIDPGFGIYKVNELVDARDVGLGAWFEAHIHSVTRASDGQSRGKTPLKNGSSCKRTNGNIKHKSKENTNKLDSVPSTSNSDCVAADEDVIYHIQYDEYPESGTLEMNVKDLRPRARTILKWNELNVGDVVMVNYNVESPGQRGFWFDAEITTLKTISRTKKELRVKIFLGGSEGTLNDCKIISVDEIFKIERPGAHPLSFADGKFLRRNDPECDLCGGDPEKKCHSCSCRVCGGKHEPNMQLLCDECNVAYHIYCLNPPLDKVPEEEYWYCPSCKTDSSEVVKAGERLKMSKKKAKMPSASTESRRDWGRGMACVGRTRECTIVPSNHYGPIPGIPVGSTWRFRVQVSEAGVHRPHVGGIHGRSNDGAYSLVLAGGFADEVDRGDEFTYTGSGGKNLAGNKRIGAPSADQTLTNMNRALALNCDAPLDDKIGAESRNWRAGKPVRVIRSFKGRKISKYAPEEGNRYDGIYKVVKYWPEISSSHGFLVWRYLLRRDDVEPAPWTSEGIERSRRLCLRLQYPAGYPSDKEGKKPKGQSKKQPSGTTKRPISDDDCPSASKVYKASDSAEAIEAFQLTPQQQHLIREDCQNQKLWDEVLSHLVEGPNFLKKLEQSFMCVCCQELVYQPVTTECFHNVCKDCLQRSFKAQVFSCPACRHDLGQNYIMIPNEILQTLLDLFFPGYSKGR",
# UNK_HUMAN
"MSKGPGPGGSAASSAPPAATAQVLQAQPEKPQHYTYLKEFRTEQCPLFVQHKCTQHRPYTCFHWHFVNQRRRRSIRRRDGTFNYSPDVYCTKYDEATGLCPEGDECPFLHRTTGDTERRYHLRYYKTGICIHETDSKGNCTKNGLHCAFAHGPHDLRSPVYDIRELQAMEALQNGQTTVEGSIEGQSAGAASHAMIEKILSEEPRWQETAYVLGNYKTEPCKKPPRLCRQGYACPYYHNSKDRRRSPRKHKYRSSPCPNVKHGDEWGDPGKCENGDACQYCHTRTEQQFHPEIYKSTKCNDMQQSGSCPRGPFCAFAHVEQPPLSDDLQPSSAVSSPTQPGPVLYMPSAAGDSVPVSPSSPHAPDLSALLCRNSSLGSPSNLCGSPPGSIRKPPNLEGIVFPGESGLAPGSYKKAPGFEREDQVGAEYLKNFKCQAKLKPHSLEPRSQEQPLLQPKQDMLGILPAGSPLTSSISSSITSSLAATPPSPVGTSSVPGMNANALPFYPTSDTVESVIESALDDLDLNEFGVAALEKTFDNSTVPHPGSITIGGSLLQSSAPVNIPGSLGSSASFHSASPSPPVSLSSHFLQQPQGHLSQSENTFLGTSASHGSLGLNGMNSSIWEHFASGSFSPGTSPAFLSGPGAAELARLRQELDEANSTIKQWEESWKQAKQACDAWKKEAEEAGERASAAGAECELAREQRDALEVQVKKLQEELERLHAGPEPQALPAFSDLEALSLSTLYSLQKQLRAHLEQVDKAVFHMQSVKCLKCQEQKRAVLPCQHAALCELCAEGSECPICQPGRAHTLQS",
# UNKL_HUMAN
"MPSVSKAAAAALSGSPPQTEKPTHYRYLKEFRTEQCPLFSQHKCAQHRPFTCFHWHFLNQRRRRPLRRRDGTFNYSPDVYCSKYNEATGVCPDGDECPYLHRTTGDTERKYHLRYYKTGTCIHETDARGHCVKNGLHCAFAHGPLDLRPPVCDVRELQAQEALQNGQLGGGEGVPDLQPGVLASQAMIEKILSEDPRWQDANFVLGSYKTEQCPKPPRLCRQGYACPHYHNSRDRRRNPRRFQYRSTPCPSVKHGDEWGEPSRCDGGDGCQYCHSRTEQQFHPESTKCNDMRQTGYCPRGPFCAFAHVEKSLGMVNEWGCHDLHLTSPSSTGSGQPGNAKRRDSPAEGGPRGSEQDSKQNHLAVFAAVHPPAPSVSSSVASSLASSAGSGSSSPTALPAPPARALPLGPASSTVEAVLGSALDLHLSNVNIASLEKDLEEQDGHDLGAAGPRSLAGSAPVAIPGSLPRAPSLHSPSSASTSPLGSLSQPLPGPVGSSAMTPPQQPPPLRSEPGTLGSAASSYSPLGLNGVPGSIWDFVSGSFSPSPSPILSAGPPSSSSASPNGAELARVRRQLDEAKRKIRQWEESWQQVKQVCDAWQREAQEAKERARVADSDRQLALQKKEEVEAQVIFQLRAKQCVACRERAHGAVLRPCQHHILCEPCAATAPECPYCKGQPLQW",
# VPS11_HUMAN
"MAAYLQWRRFVFFDKELVKEPLSNDGAAPGATPASGSAASKFLCLPPGITVCDSGRGSLVFGDMEGQIWFLPRSLQLTGFQAYKLRVTHLYQLKQHNILASVGEDEEGINPLVKIWNLEKRDGGNPLCTRIFPAIPGTEPTVVSCLTVHENLNFMAIGFTDGSVTLNKGDITRDRHSKTQILHKGNYPVTGLAFRQAGKTTHLFVVTTENVQSYIVSGKDYPRVELDTHGCGLRCSALSDPSQDLQFIVAGDECVYLYQPDERGPCFAFEGHKLIAHWFRGYLIIVSRDRKVSPKSEFTSRDSQSSDKQILNIYDLCNKFIAYSTVFEDVVDVLAEWGSLYVLTRDGRVHALQEKDTQTKLEMLFKKNLFEMAINLAKSQHLDSDGLAQIFMQYGDHLYSKGNHDGAVQQYIRTIGKLEPSYVIRKFLDAQRIHNLTAYLQTLHRQSLANADHTTLLLNCYTKLKDSSKLEEFIKKKSESEVHFDVETAIKVLRQAGYYSHALYLAENHAHHEWYLKIQLEDIKNYQEALRYIGKLPFEQAESNMKRYGKILMHHIPEQTTQLLKGLCTDYRPSLEGRSDREAPGCRANSEEFIPIFANNPRELKAFLEHMSEVQPDSPQGIYDTLLELRLQNWAHEKDPQVKEKLHAEAISLLKSGRFCDVFDKALVLCQMHDFQDGVLYLYEQGKLFQQIMHYHMQHEQYRQVISVCERHGEQDPSLWEQALSYFARKEEDCKEYVAAVLKHIENKNLMPPLLVVQTLAHNSTATLSVIRDYLVQKLQKQSQQIAQDELRVRRYREETTRIRQEIQELKASPKIFQKTKCSICNSALELPSVHFLCGHSFHQHCFESYSESDADCPTCLPENRKVMDMIRAQEQKRDLHDQFQHQLRCSNDSFSVIADYFGRGVFNKLTLLTDPPTARLTSSLEAGLQRDLLMHSRRGT",
# VPS18_HUMAN
"MASILDEYENSLSRSAVLQPGCPSVGIPHSGYVNAQLEKEVPIFTKQRIDFTPSERITSLVVSSNQLCMSLGKDTLLRIDLGKANEPNHVELGRKDDAKVHKMFLDHTGSHLLIALSSTEVLYVNRNGQKVRPLARWKGQLVESVGWNKALGTESSTGPILVGTAQGHIFEAELSASEGGLFGPAPDLYFRPLYVLNEEGGPAPVCSLEAERGPDGRSFVIATTRQRLFQFIGRAAEGAEAQGFSGLFAAYTDHPPPFREFPSNLGYSELAFYTPKLRSAPRAFAWMMGDGVLYGALDCGRPDSLLSEERVWEYPEGVGPGASPPLAIVLTQFHFLLLLADRVEAVCTLTGQVVLRDHFLEKFGPLKHMVKDSSTGQLWAYTERAVFRYHVQREARDVWRTYLDMNRFDLAKEYCRERPDCLDTVLAREADFCFRQRRYLESARCYALTQSYFEEIALKFLEARQEEALAEFLQRKLASLKPAERTQATLLTTWLTELYLSRLGALQGDPEALTLYRETKECFRTFLSSPRHKEWLFASRASIHELLASHGDTEHMVYFAVIMQDYERVVAYHCQHEAYEEALAVLARHRDPQLFYKFSPILIRHIPRQLVDAWIEMGSRLDARQLIPALVNYSQGGEVQQVSQAIRYMEFCVNVLGETEQAIHNYLLSLYARGRPDSLLAYLEQAGASPHRVHYDLKYALRLCAEHGHHRACVHVYKVLELYEEAVDLALQVDVDLAKQCADLPEEDEELRKKLWLKIARHVVQEEEDVQTAMACLASCPLLKIEDVLPFFPDFVTIDHFKEAICSSLKAYNHHIQELQREMEEATASAQRIRRDLQELRGRYGTVEPQDKCATCDFPLLNRPFYLFLCGHMFHADCLLQAVRPGLPAYKQARLEELQRKLGAAPPPAKGSARAKEAEGGAATAGPSREQLKADLDELVAAECVYCGELMIRSIDRPFIDPQRYEEEQLSWL",
# VPS41_HUMAN
"MAEAEEQETGSLEESTDESEEEESEEEPKLKYERLSNGVTEILQKDAASCMTVHDKFLALGTHYGKVYLLDVQGNITQKFDVSPVKINQISLDESGEHMGVCSEDGKVQVFGLYSGEEFHETFDCPIKIIAVHPHFVRSSCKQFVTGGKKLLLFERSWMNRWKSAVLHEGEGNIRSVKWRGHLIAWANNMGVKIFDIISKQRITNVPRDDISLRPDMYPCSLCWKDNVTLIIGWGTSVKVCSVKERHASEMRDLPSRYVEIVSQFETEFYISGLAPLCDQLVVLSYVKEISEKTEREYCARPRLDIIQPLSETCEEISSDALTVRGFQENECRDYHLEYSEGESLFYIVSPRDVVVAKERDQDDHIDWLLEKKKYEEALMAAEISQKNIKRHKILDIGLAYINHLVERGDYDIAARKCQKILGKNAALWEYEVYKFKEIGQLKAISPYLPRGDPVLKPLIYEMILHEFLESDYEGFATLIREWPGDLYNNSVIVQAVRDHLKKDSQNKTLLKTLAELYTYDKNYGNALEIYLTLRHKDVFQLIHKHNLFSSIKDKIVLLMDFDSEKAVDMLLDNEDKISIKKVVEELEDRPELQHVYLHKLFKRDHHKGQRYHEKQISLYAEYDRPNLLPFLRDSTHCPLEKALEICQQRNFVEETVYLLSRMGNSRSALKMIMEELHDVDKAIEFAKEQDDGELWEDLILYSIDKPPFITGLLNNIGTHVDPILLIHRIKEGMEIPNLRDSLVKILQDYNLQILLREGCKKILVADSLSLLKKMHRTQMKGVLVDEENICESCLSPILPSDAAKPFSVVVFHCRHMFHKECLPMPSMNSAAQFCNICSAKNRGPGSAILEMKK",
# VPS8_HUMAN
"MENEPDHENVEQSLCAKTSEEELNKSFNLEASLSKFSYIDMDKELEFKNDLIDDKEFDIPQVDTPPTLESILNETDDEDESFILEDPTLLNIDTIDSHSYDTSSVASSDSGDRTNLKRKKKLPDSFSLHGSVMRHSLLKGISAQIVSAADKVDAGLPTAIAVSSLIAVGTSHGLALIFGKDQNQALRLCLGSTSVGGQYGAISALSINNDCSRLLCGFAKGQITMWDLASGKLLRSITDAHPPGTAILHIKFTDDPTLAICNDSGGSVFELTFKRVMGVRTCESRCLFSGSKGEVCCIEPLHSKPELKDHPITQFSLLAMASLTKILVIGLKPSLKVWMTFPYGRMDPSSVPLLAWHFVAVQNYVNPMLAFCRGDVVHFLLVKRDESGAIHVTKQKHLHLYYDLINFTWINSRTVVLLDSVEKLHVIDRQTQEELETVEISEVQLVYNSSHFKSLATGGNVSQALALVGEKACYQSISSYGGQIFYLGTKSVYVMMLRSWRERVDHLLKQDCLTEALALAWSFHEGKAKAVVGLSGDASKRKAIVADRMVEILFHYADRALKKCPDQGKIQVMEQHFQDMVPVIVDYCLLLQRKDLLFSQMYDKLSENSVAKGVFLECLEPYILSDKLVGITPQVMKDLIVHFQDKKLMENVEALIVHMDITSLDIQQVVLMCWENRLYDAMIYVYNRGMNEFISPMEKLFRVIAPPLNAGKTLTDEQVVMGNKLLVYISCCLAGRAYPLGDIPEDLVPLVKNQVFEFLIRLHSAEASPEEEIYPYIRTLLHFDTREFLNVLALTFEDFKNDKQAVEYQQRIVDILLKVMVENSDFTPSQVGCLFTFLARQLAKPDNTLFVNRTLFDQVLEFLCSPDDDSRHSERQQVLLELLQAGGIVQFEESRLIRMAEKAEFYQICEFMYEREHQYDKIIDCYLRDPLREEEVFNYIHNILSIPGHSAEEKQSVWQKAMDHIEELVSLKPCKAAELVATHFSGHIETVIKKLQNQVLLFKFLRSLLDPREGIHVNQELLQISPCITEQFIELLCQFNPTQVIETLQVLECYRLEETIQITQKYQLHEVTAYLLEKKGDIHGAFLIMLERLQSKLQEVTHQGENTKEDPSLKDVEDTMVETIALCQRNSHNLNQQQREALWFPLLEAMMAPQKLSSSAIPHLHSEALKSLTMQVLNSMAAFIALPSILQRILQDPVYGKGKLGEIQGLILGMLDTFNYEQTLLETTTSLLNQDLHWSLCNLRASVTRGLNPKQDYCSICLQQYKRRQEMADEIIVFSCGHLYHSFCLQNKECTVEFEGQTRWTCYKCSSSNKVGKLSENSSEIKKGRITPSQVKMSPSYHQSKGDPTAKKGTSEPVLDPQQIQAFDQLCRLYRGSSRLALLTELSQNRSSESYRPFSGSQSAPAFNSIFQNENFQLQLIPPPVTED",
# WDR59_HUMAN
"MAARWSSENVVVEFRDSQATAMSVDCLGQHAVLSGRRFLYIVNLDAPFEGHRKISRQSKWDIGAVQWNPHDSFAHYFAASSNQRVDLYKWKDGSGEVGTTLQGHTRVISDLDWAVFEPDLLVTSSVDTYIYIWDIKDTRKPTVALSAVAGASQVKWNKKNANCLATSHDGDVRIWDKRKPSTAVEYLAAHLSKIHGLDWHPDSEHILATSSQDNSVKFWDYRQPRKYLNILPCQVPVWKARYTPFSNGLVTVMVPQLRRENSLLLWNVFDLNTPVHTFVGHDDVVLEFQWRKQKEGSKDYQLVTWSRDQTLRMWRVDSQMQRLCANDILDGVDEFIESISLLPEPEKTLHTEDTDHQHTASHGEEEALKEDPPRNLLEERKSDQLGLPQTLQQEFSLINVQIRNVNVEMDAADRSCTVSVHCSNHRVKMLVKFPAQYPNNAAPSFQFINPTTITSTMKAKLLKILKDTALQKVKRGQSCLEPCLRQLVSCLESFVNQEDSASSNPFALPNSVTPPLPTFARVTTAYGSYQDANIPFPRTSGARFCGAGYLVYFTRPMTMHRAVSPTEPTPRSLSALSAYHTGLIAPMKIRTEAPGNLRLYSGSPTRSEKEQVSISSFYYKERKSRRWKSKREGSDSGNRQIKAAGKVIIQDIACLLPVHKSLGELYILNVNDIQETCQKNAASALLVGRKDLVQVWSLATVATDLCLGPKSDPDLETPWARHPFGRQLLESLLAHYCRLRDVQTLAMLCSVFEAQSRPQGLPNPFGPFPNRSSNLVVSHSRYPSFTSSGSCSSMSDPGLNTGGWNIAGREAEHLSSPWGESSPEELRFGSLTYSDPRERERDQHDKNKRLLDPANTQQFDDFKKCYGEILYRWGLREKRAEVLKFVSCPPDPHKGIEFGVYCSHCRSEVRGTQCAICKGFTFQCAICHVAVRGSSNFCLTCGHGGHTSHMMEWFRTQEVCPTGCGCHCLLESTF",
# WSDU1_HUMAN
"MVKLIHTLADHGDDVNCCAFSFSLLATCSLDKTIRLYSLRDFTELPHSPLKFHTYAVHCCCFSPSGHILASCSTDGTTVLWNTENGQMLAVMEQPSGSPVRVCQFSPDSTCLASGAADGTVVLWNAQSYKLYRCGSVKDGSLAACAFSPNGSFFVTGSSCGDLTVWDDKMRCLHSEKAHDLGITCCDFSSQPVSDGEQGLQFFRLASCGQDCQVKIWIVSFTHILGFELKYKSTLSGHCAPVLACAFSHDGQMLVSGSVDKSVIVYDTNTENILHTLTQHTRYVTTCAFAPNTLLLATGSMDKTVNIWQFDLETLCQARRTEHQLKQFTEDWSEEDVSTWLCAQDLKDLVGIFKMNNIDGKELLNLTKESLADDLKIESLGLRSKVLRKIEELRTKVKSLSSGIPDEFICPITRELMKDPVIASDGYSYEKEAMENWISKKKRTSPMTNLVLPSAVLTPNRTLKMAINRWLETHQK",
# WWP1_HUMAN
"MATASPRSDTSNNHSGRLQLQVTVSSAKLKRKKNWFGTAIYTEVVVDGEITKTAKSSSSSNPKWDEQLTVNVTPQTTLEFQVWSHRTLKADALLGKATIDLKQALLIHNRKLERVKEQLKLSLENKNGIAQTGELTVVLDGLVIEQENITNCSSSPTIEIQENGDALHENGEPSARTTARLAVEGTNGIDNHVPTSTLVQNSCCSYVVNGDNTPSSPSQVAARPKNTPAPKPLASEPADDTVNGESSSFAPTDNASVTGTPVVSEENALSPNCTSTTVEDPPVQEILTSSENNECIPSTSAELESEARSILEPDTSNSRSSSAFEAAKSRQPDGCMDPVRQQSGNANTETLPSGWEQRKDPHGRTYYVDHNTRTTTWERPQPLPPGWERRVDDRRRVYYVDHNTRTTTWQRPTMESVRNFEQWQSQRNQLQGAMQQFNQRYLYSASMLAAENDPYGPLPPGWEKRVDSTDRVYFVNHNTKTTQWEDPRTQGLQNEEPLPEGWEIRYTREGVRYFVDHNTRTTTFKDPRNGKSSVTKGGPQIAYERGFRWKLAHFRYLCQSNALPSHVKINVSRQTLFEDSFQQIMALKPYDLRRRLYVIFRGEEGLDYGGLAREWFFLLSHEVLNPMYCLFEYAGKNNYCLQINPASTINPDHLSYFCFIGRFIAMALFHGKFIDTGFSLPFYKRMLSKKLTIKDLESIDTEFYNSLIWIRDNNIEECGLEMYFSVDMEILGKVTSHDLKLGGSNILVTEENKDEYIGLMTEWRFSRGVQEQTKAFLDGFNEVVPLQWLQYFDEKELEVMLCGMQEVDLADWQRNTVYRHYTRNSKQIIWFWQFVKETDNEVRMRLLQFVTGTCRLPLGGFAELMGSNGPQKFCIEKVGKDTWLPRSHTCFNRLDLPPYKSYEQLKEKLLFAIEETEGFGQE",
# WWP2_HUMAN
"MASASSSRAGVALPFEKSQLTLKVVSAKPKVHNRQPRINSYVEVAVDGLPSETKKTGKRIGSSELLWNEIIILNVTAQSHLDLKVWSCHTLRNELLGTASVNLSNVLKNNGGKMENMQLTLNLQTENKGSVVSGGELTIFLDGPTVDLGNVPNGSALTDGSQLPSRDSSGTAVAPENRHQPPSTNCFGGRSRTHRHSGASARTTPATGEQSPGARSRHRQPVKNSGHSGLANGTVNDEPTTATDPEEPSVVGVTSPPAAPLSVTPNPNTTSLPAPATPAEGEEPSTSGTQQLPAAAQAPDALPAGWEQRELPNGRVYYVDHNTKTTTWERPLPPGWEKRTDPRGRFYYVDHNTRTTTWQRPTAEYVRNYEQWQSQRNQLQGAMQHFSQRFLYQSSSASTDHDPLGPLPPGWEKRQDNGRVYYVNHNTRTTQWEDPRTQGMIQEPALPPGWEMKYTSEGVRYFVDHNTRTTTFKDPRPGFESGTKQGSPGAYDRSFRWKYHQFRFLCHSNALPSHVKISVSRQTLFEDSFQQIMNMKPYDLRRRLYIIMRGEEGLDYGGIAREWFFLLSHEVLNPMYCLFEYAGKNNYCLQINPASSINPDHLTYFRFIGRFIAMALYHGKFIDTGFTLPFYKRMLNKRPTLKDLESIDPEFYNSIVWIKENNLEECGLELYFIQDMEILGKVTTHELKEGGESIRVTEENKEEYIMLLTDWRFTRGVEEQTKAFLDGFNEVAPLEWLRYFDEKELELMLCGMQEIDMSDWQKSTIYRHYTKNSKQIQWFWQVVKEMDNEKRIRLLQFVTGTCRLPVGGFAELIGSNGPQKFCIDKVGKETWLPRSHTCFNRLDLPPYKSYEQLREKLLYAIEETEGFGQE",
# XIAP_HUMAN
"MTFNSFEGSKTCVPADINKEEEFVEEFNRLKTFANFPSGSPVSASTLARAGFLYTGEGDTVRCFSCHAAVDRWQYGDSAVGRHRKVSPNCRFINGFYLENSATQSTNSGIQNGQYKVENYLGSRDHFALDRPSETHADYLLRTGQVVDISDTIYPRNPAMYSEEARLKSFQNWPDYAHLTPRELASAGLYYTGIGDQVQCFCCGGKLKNWEPCDRAWSEHRRHFPNCFFVLGRNLNIRSESDAVSSDRNFPNSTNLPRNPSMADYEARIFTFGTWIYSVNKEQLARAGFYALGEGDKVKCFHCGGGLTDWKPSEDPWEQHAKWYPGCKYLLEQKGQEYINNIHLTHSLEECLVRTTEKTPSLTRRIDDTIFQNPMVQEAIRMGFSFKDIKKIMEEKIQISGSNYKSLEVLVADLVNAQKDSMQDESSQTSLQKEISTEEQLRRLQEEKLCKICMDRNIAIVFVPCGHLVTCKQCAEAVDKCPMCYTVITFKQKIFMS",
# ZBT12_HUMAN
"MASGVEVLRFQLPGHEAATLRNMNQLRAEERFCDVTIVADSLKFRGHKVILAACSPFLRDQFLLNPSSELQVSLMHSARIVADLLLSCYTGALEFAVRDIVNYLTAASYLQMEHVVEKCRNALSQFIEPKIGLKEDGVSEASLVSSISATKSLLPPARTPKPAPKPPPPPPLPPPLLRPVKLEFPLDEDLELKAEEEDEDEDEDVSDICIVKVESALEVAHRLKPPGGLGGGLGIGGSVGGHLGELAQSSVPPSTVAPPQGVVKACYSLSEDAEGEGLLLIPGGRASVGATSGLVEAAAVAMAARGAGGSLGAGGSRGPLPGGFSGGNPLKNIKCTKCPEVFQGVEKLVFHMRAQHFIFMCPRCGKQFNHSSNLNRHMNVHRGVKSHSCGICGKCFTQKSTLHDHLNLHSGARPYRCSYCDVRFAHKPAIRRHLKEQHGKTTAENVLEASVAEINVLIR",
# ZFP91_HUMAN
"MPGETEEPRPPEQQDQEGGEAAKAAPEEPQQRPPEAVAAAPAGTTSSRVLRGGRDRGRAAAAAAAAAVSRRRKAEYPRRRRSSPSARPPDVPGQQPQAAKSPSPVQGKKSPRLLCIEKVTTDKDPKEEKEEEDDSALPQEVSIAASRPSRGWRSSRTSVSRHRDTENTRSSRSKTGSLQLICKSEPNTDQLDYDVGEEHQSPGGISSEEEEEEEEEMLISEEEIPFKDDPRDETYKPHLERETPKPRRKSGKVKEEKEKKEIKVEVEVEVKEEENEIREDEEPPRKRGRRRKDDKSPRLPKRRKKPPIQYVRCEMEGCGTVLAHPRYLQHHIKYQHLLKKKYVCPHPSCGRLFRLQKQLLRHAKHHTDQRDYICEYCARAFKSSHNLAVHRMIHTGEKPLQCEICGFTCRQKASLNWHMKKHDADSFYQFSCNICGKKFEKKDSVVAHKAKSHPEVLIAEALAANAGALITSTDILGTNPESLTQPSDGQGLPLLPEPLGNSTSGECLLLEAEGMSKSYCSGTERVSLMADGKIFVGSGSSGGTEGLVMNSDILGATTEVLIEDSDSAGP",
# ZFPL1_HUMAN
"MGLCKCPKRKVTNLFCFEHRVNVCEHCLVANHAKCIVQSYLQWLQDSDYNPNCRLCNIPLASRETTRLVCYDLFHWACLNERAAQLPRNTAPAGYQCPSCNGPIFPPTNLAGPVASALREKLATVNWARAGLGLPLIDEVVSPEPEPLNTSDFSDWSSFNASSTPGPEEVDSASAAPAFYSQAPRPPASPGRPEQHTVIHMGNPEPLTHAPRKVYDTRDDDRTPGLHGDCDDDKYRRRPALGWLARLLRSRAGSRKRPLTLLQRAGLLLLLGLLGFLALLALMSRLGRAAADSDPNLDPLMNPHIRVGPS",
# Z280A_HUMAN
"MGDIFLCKKVESPKKNLRESKQREEDDEDPDLIYVGVEHVHRDAEVLFVGMISNSKPVVSNILNRVTPGSNSRRKKGHFRQYPAHVSQPANHVTSMAKAIMPVSLSEGRSTDSPVTMKSSSEPGYKMSSPQVVSPSSSDSLPPGTQCLVGAMVSGGGRNESSPDSKRLSTSDINSRDSKRVKLRDGIPGVPSLAVVPSDMSSTISTNTPSQGICNSSNHVQNGVTFPWPDANGKAHFNLTDPERASESALAMTDISSLASQNKTFDPKKENPIVLLSDFYYGQHKGDGQPEQKTHTTFKCLSCVKVLKNIKFMNHMKHHLEFEKQRNDSWEDHTTCQHCHRQFPTPFQLQCHIDSVHIAMGPSAVCKICELSFETDQVLLQHMKDHHKPGEMPYVCQVCHYRSSVFADVETHFRTCHENTKNLLCLFCLKLFKTAIPYMNHCWRHSRRRVLQCSKCRLQFLTLKEEIEHKTKDHQTFKKPEQLQGLPSETKVIIQTSVQPGSSGMASVIVSNTDPQSSPVKTKKKTAMNTRDSRLPCSKDSS",
# ZN341_HUMAN
"MAQAIFEALEGMDNQTVLAVQSLLDGQGAVPDPTGQSVNAPPAIQPLDDEDVFLCGKCKKQFNSLPAFMTHKREQCQGNAPALATVSLATNSIYPPSAAPTAVQQAPTPANRQISTYITVPPSPLIQTLVQGNILVSDDVLMSAMSAFTSLDQPMPQGPPPVQSSLNMHSVPSYLTQPPPPPPPPPPLPPPPPPQPPPPPPQSLGPPGRPNPGGNGVVEVYSAAAPLAGSGTVEIQALGMQPYPPLEVPNQCVEPPVYPTPTVYSPGKQGFKPKGPNPAAPMTSATGGTVATFDSPATLKTRRAKGARGLPEAAGKPKAQKLKCSYCDKSFTKNFDLQQHIRSHTGEKPFQCIACGRAFAQKSNVKKHMQTHKVWPPGHSGGTVSRNSVTVQVMALNPSRQEDEESTGLGQPLPGAPQPQALSTAGEEEGDKPESKQVVLIDSSYLCQFCPSKFSTYFQLKSHMTQHKNEQVYKCVVKSCAQTFPKLDTFLEHIKSHQEELSYRCHLCGKDFPSLYDLGVHQYSHSLLPQHSPKKDNAVYKCVKCVNKYSTPEALEHHLQTATHNFPCPHCQKVFPCERYLRRHLPTHGSGGRFKCQVCKKFFRREHYLKLHAHIHSGEKPYKCSVCESAFNRKDKLKRHMLIHEPFKKYKCPFSTHTGCSKEFNRPDKLKAHILSHSGMKLHKCALCSKSFSRRAHLAEHQRAHTGNYKFRCAGCAKGFSRHKYLKDHRCRLGPQKDKDLQTRRPPQRRAAPRSCGSGGRKVLTPLPDPLGLEELKDTGAGLVPEAVPGKPPFAEPDAVLSIVVGGAVGAETELVVPGHAEGLGSNLALAELQAGAEGPCAMLAVPVYIQASE",
# ZN511_HUMAN
"MQLPPALCARLAAGPGAAEPLPVERDPAAGAAPFRFVARPVRFPREHQFFEDGDVQRHLYLQDVIMQVADVPEKPRVPAFACQVAGCCQVFDALDDYEHHYHTLHGNVCSFCKRAFPSGHLLDAHILEWHDSLFQILSERQDMYQCLVEGCTEKFKTSRDRKDHMVRMHLYPADFRFDKPKKSRSPASAEAPGDSGERSEGEAMEICSEPVAASPAPAGERRIYRHRIPSTICFGQGAARGFKSNKKKTKQC",
# ZN521_HUMAN
"MSRRKQAKPRSLKDPNCKLEDKTEDGEALDCKKRPEDGEELEDEAVHSCDSCLQVFESLSDITEHKINQCQLTDGVDVEDDPTCSWPASSPSSKDQTSPSHGEGCDFGEEEGGPGLPYPCQFCDKSFSRLSYLKHHEQSHSDKLPFKCTYCSRLFKHKRSRDRHIKLHTGDKKYHCSECDAAFSRSDHLKIHLKTHTSNKPYKCAICRRGFLSSSSLHGHMQVHERNKDGSQSGSRMEDWKMKDTQKCSQCEEGFDFPEDLQKHIAECHPECSPNEDRAALQCVYCHELFVEETSLMNHMEQVHSGEKKNSCSICSESFHTVEELYSHMDSHQQPESCNHSNSPSLVTVGYTSVSSTTPDSNLSVDSSTMVEAAPPIPKSRGRKRAAQQTPDMTGPSSKQAKVTYSCIYCNKQLFSSLAVLQIHLKTMHLDKPEQAHICQYCLEVLPSLYNLNEHLKQVHEAQDPGLIVSAMPAIVYQCNFCSEVVNDLNTLQEHIRCSHGFANPAAKDSNAFFCPHCYMGFLTDSSLEEHIRQVHCDLSGSRFGSPVLGTPKEPVVEVYSCSYCTNSPIFNSVLKLNKHIKENHKNIPLALNYIHNGKKSRALSPLSPVAIEQTSLKMMQAVGGAPARPTGEYICNQCGAKYTSLDSFQTHLKTHLDTVLPKLTCPQCNKEFPNQESLLKHVTIHFMITSTYYICESCDKQFTSVDDLQKHLLDMHTFVFFRCTLCQEVFDSKVSIQLHLAVKHSNEKKVYRCTSCNWDFRNETDLQLHVKHNHLENQGKVHKCIFCGESFGTEVELQCHITTHSKKYNCKFCSKAFHAIILLEKHLREKHCVFETKTPNCGTNGASEQVQKEEVELQTLLTNSQESHNSHDGSEEDVDTSEPMYGCDICGAAYTMETLLQNHQLRDHNIRPGESAIVKKKAELIKGNYKCNVCSRTFFSENGLREHMQTHLGPVKHYMCPICGERFPSLLTLTEHKVTHSKSLDTGNCRICKMPLQSEEEFLEHCQMHPDLRNSLTGFRCVVCMQTVTSTLELKIHGTFHMQKTGNGSAVQTTGRGQHVQKLYKCASCLKEFRSKQDLVKLDINGLPYGLCAGCVNLSKSASPGINVPPGTNRPGLGQNENLSAIEGKGKVGGLKTRCSSCNVKFESESELQNHIQTIHRELVPDSNSTQLKTPQVSPMPRISPSQSDEKKTYQCIKCQMVFYNEWDIQVHVANHMIDEGLNHECKLCSQTFDSPAKLQCHLIEHSFEGMGGTFKCPVCFTVFVQANKLQQHIFSAHGQEDKIYDCTQCPQKFFFQTELQNHTMTQHSS",
# ZN598_HUMAN
"MAAAGGAEGRRAALEAAAAAAPERGGGSCVLCCGDLEATALGRCDHPVCYRCSTKMRVLCEQRYCAVCREELRQVVFGKKLPAFATIPIHQLQHEKKYDIYFADGKVYALYRQLLQHECPRCPELPPFSLFGDLEQHMRRQHELFCCRLCLQHLQIFTYERKWYSRKDLARHRMQGDPDDTSHRGHPLCKFCDERYLDNDELLKHLRRDHYFCHFCDSDGAQDYYSDYAYLREHFREKHFLCEEGRCSTEQFTHAFRTEIDLKAHRTACHSRSRAEARQNRHIDLQFSYAPRHSRRNEGVVGGEDYEEVDRYSRQGRVARAGTRGAQQSRRGSWRYKREEEDREVAAAVRASVAAQQQEEARRSEDQEEGGRPKKEEAAARGPEDPRGPRRSPRTQGEGPGPKETSTNGPVSQEAFSVTGPAAPGCVGVPGALPPPSPKLKDEDFPSLSASTSSSCSTAATPGPVGLALPYAIPARGRSAFQEEDFPALVSSVPKPGTAPTSLVSAWNSSSSSKKVAQPPLSAQATGSGQPTRKAGKGSRGGRKGGPPFTQEEEEDGGPALQELLSTRPTGSVSSTLGLASIQPSKVGKKKKVGSEKPGTTLPQPPPATCPPGALQAPEAPASRAEGPVAVVVNGHTEGPAPARSAPKEPPGLPRPLGSFPCPTPQEDFPALGGPCPPRMPPPPGFSAVVLLKGTPPPPPPGLVPPISKPPPGFSGLLPSPHPACVPSPATTTTTKAPRLLPAPRAYLVPENFRERNLQLIQSIRDFLQSDEARFSEFKSHSGEFRQGLISAAQYYKSCRDLLGENFQKVFNELLVLLPDTAKQQELLSAHTDFCNREKPLSTKSKKNKKSAWQATTQQAGLDCRVCPTCQQVLAHGDASSHQALHAARDDDFPSLQAIARIIT",
# CBLL2_HUMAN
"MNKMPAGEQECEYNKEGKYYSKGVKLVRKKKKIPGYRWGDIKINIIGEKDDLPIHFCDKCDLPIKIYGRIIPCKHAFCYHCANLYDKVGYKVCPRCRYPVLRIEAHKRGSVFMCSIVQQCKRTYLSQKSLQAHIKRRHKRARKQVTSASLEKVRPHIAPPQTEISDIPKRLQDRDHLSYIPPEQHTMVSLPSVQHMLQEQHNQPHKDIQAPPPELSLSLPFPIQWETVSIFTRKHGNLTVDHIQNNSDSGAKKPTPPDYYPECQSQPAVSSPHHIIPQKQHYAPPPSPSSPVNHQMPYPPQDVVTPNSVRSQVPALTTTYDPSSGYIIVKVPPDMNSPPLRAPQSQNGNPSASEFASHHYNLNILPQFTENQETLSPQFTQTDAMDHRRWPAWKRLSPCPPTRSPPPSTLHGRSHHSHQRRHRRY",
# ZNRF1_HUMAN
"MGGKQSTAARSRGPFPGVSTDDSAVPPPGGAPHFGHYRTGGGAMGLRSRSVSSVAGMGMDPSTAGGVPFGLYTPASRGTGDSERAPGGGGSASDSTYAHGNGYQETGGGHHRDGMLYLGSRASLADALPLHIAPRWFSSHSGFKCPICSKSVASDEMEMHFIMCLSKPRLSYNDDVLTKDAGECVICLEELLQGDTIARLPCLCIYHKSCIDSWFEVNRSCPEHPAD",
# ZNRF2_HUMAN
"MGAKQSGPAAANGRTRAYSGSDLPSSSSGGANGTAGGGGGARAAAAGRFPAQVPSAHQPSASGGAAAAAAAPAAPAAPRSRSLGGAVGSVASGARAAQSPFSIPNSSSGPYGSQDSVHSSPEDGGGGRDRPVGGSPGGPRLVIGSLPAHLSPHMFGGFKCPVCSKFVSSDEMDLHLVMCLTKPRITYNEDVLSKDAGECAICLEELQQGDTIARLPCLCIYHKGCIDEWFEVNRSCPEHPSD",
# ZNRF3_HUMAN
"MRPRSGGRPGATGRRRRRLRRRPRGLRCSRLPPPPPLPLLLGLLLAAAGPGAARAKETAFVEVVLFESSPSGDYTTYTTGLTGRFSRAGATLSAEGEIVQMHPLGLCNNNDEEDLYEYGWVGVVKLEQPELDPKPCLTVLGKAKRAVQRGATAVIFDVSENPEAIDQLNQGSEDPLKRPVVYVKGADAIKLMNIVNKQKVARARIQHRPPRQPTEYFDMGIFLAFFVVVSLVCLILLVKIKLKQRRSQNSMNRLAVQALEKMETRKFNSKSKGRREGSCGALDTLSSSSTSDCAICLEKYIDGEELRVIPCTHRFHRKCVDPWLLQHHTCPHCRHNIIEQKGNPSAVCVETSNLSRGRQQRVTLPVHYPGRVHRTNAIPAYPTRTSMDSHGNPVTLLTMDRHGEQSLYSPQTPAYIRSYPPLHLDHSLAAHRCGLEHRAYSPAHPFRRPKLSGRSFSKAACFSQYETMYQHYYFQGLSYPEQEGQSPPSLAPRGPARAFPPSGSGSLLFPTVVHVAPPSHLESGSTSSFSCYHGHRSVCSGYLADCPGSDSSSSSSSGQCHCSSSDSVVDCTEVSNQGVYGSCSTFRSSLSSDYDPFIYRSRSPCRASEAGGSGSSGRGPALCFEGSPPPEELPAVHSHGAGRGEPWPGPASPSGDQVSTCSLEMNYSSNSSLEHRGPNSSTSEVGLEASPGAAPDLRRTWKGGHELPSCACCCEPQPSPAGPSAGAAGSSTLFLGPHLYEGSGPAGGEPQSGSSQGLYGLHPDHLPRTDGVKYEGLPCCFYEEKQVARGGGGGSGCYTEDYSVSVQYTLTEEPPPGCYPGARDLSQRIPIIPEDVDCDLGLPSDCQGTHSLGSWGGTRGPDTPRPHRGLGATREEERALCCQARALLRPGCPPEEAGAVRANFPSALQDTQESSTTATEAAGPRSHSADSSSPGA",
# ZNRF4_HUMAN
"MPLCRPEHLMPRASRVPVAASLPLSHAVIPTQLPSRPGHRPPGRPRRCPKASCLPPPVGPSSTQTAKRVTMGWPRPGRALVAVKALLVLSLLQVPAQAVVRAVLEDNSSSVDFADLPALFGVPLAPEGIRGYLMEVKPANACHPIEAPRLGNRSLGAIVLIRRYDCTFDLKVLNAQRAGFEAAIVHNVHSDDLVSMTHVYEDLRGQIAIPSVFVSEAASQDLRVILGCNKSAHALLLPDDPPCHDLGCHPVLTVSWVLGCTLALVVSAFFVLNHLWLWAQACCSHRRPVKTSTCQKAQVRTFTWHNDLCAICLDEYEEGDQLKILPCSHTYHCKCIDPWFSQAPRRSCPVCKQSVAATEDSFDSTTYSFRDEDPSLPGHRPPIWAIQVQLRSRRLELLGRASPHCHCSTTSLEAEYTTVSSAPPEAPGQ",
# ZSWM2_HUMAN
"MLRRGYKASERRRHLSERLSWHQDQALSSSIYLLREMGPTGFLLREEEPEYMDFRVFLGNPHVCNCSTFPKGGELCKHICWVLLKKFKLPRNHESALQLGLGEREISDLLRGIHRVQTPQPGTNDENEHVEEDGYIKQKEIDSEDICSICQELLLEKKLPVTFCRFGCGNSIHIKCMKILANYQSTSNTSMLKCPLCRKEFAPLKLILEEFKNSSKLVAAAEKERLDKHLGIPCNNCKQFPIEGKCYKCTECIEYHLCQECFDSCCHLSHTFTFREKRNQKWRSLEKRADEVVKYIDTKNEIEEKMSHFQEKQGQVYTPKHIVRSLPLQLITKNSKLLAPGYQCLLCLKAFHLGQHTRLLPCTHKFHRKCIDNWLFHKCNSCPIDGQVIYNPLTWKNSAVNGQAHQSVSNRDIIHLSKQKEPDLFIPGTGLVLKQNRLGILPSIPQCNFDELNTPQSPKDAYENTTIDNLCSIKLDNSNSKKLTYDYKISQHFPRYLQDLPTVSFGKIPSQTLLPPIVHKNIVCPTAMESPCISGKFHTSLSRMTKGCKCNNHNLKKTPATKIREDNKRSTLLPEDFNLIVNWSTAKLSLSKRYSNCMGEITRKCSHLSRQPVSHSVNTKSTELSLIIEGVQL",
# ZXDC_HUMAN
"MDLPALLPAPTARGGQHGGGPGPLRRAPAPLGASPARRRLLLVRGPEDGGPGARPGEASGPSPPPAEDDSDGDSFLVLLEVPHGGAAAEAAGSQEAEPGSRVNLASRPEQGPSGPAAPPGPGVAPAGAVTISSQDLLVRLDRGVLALSAPPGPATAGAAAPRRAPQASGPSTPGYRCPEPQCALAFAKKHQLKVHLLTHGGGQGRRPFKCPLEGCGWAFTTSYKLKRHLQSHDKLRPFGCPVGGCGKKFTTVYNLKAHMKGHEQESLFKCEVCAERFPTHAKLSSHQRSHFEPERPYKCDFPGCEKTFITVSALFSHNRAHFREQELFSCSFPGCSKQYDKACRLKIHLRSHTGERPFICDSDSCGWTFTSMSKLLRHRRKHDDDRRFTCPVEGCGKSFTRAEHLKGHSITHLGTKPFECPVEGCCARFSARSSLYIHSKKHVQDVGAPKSRCPVSTCNRLFTSKHSMKAHMVRQHSRRQDLLPQLEAPSSLTPSSELSSPGQSELTNMDLAALFSDTPANASGSAGGSDEALNSGILTIDVTSVSSSLGGNLPANNSSLGPMEPLVLVAHSDIPPSLDSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDPLALTSNSNLAAHITTPTSSSTPRENASVPELLAPIKVEPDSPSRPGAVGQQEGSHGLPQSTLPSPAEQHGAQDTELSAGTGNFYLESGGSARTDYRAIQLAKEKKQRGAGSNAGASQSTQRKIKEGKMSPPHFHASQNSWLCGSLVVPSGGRPGPAPAAGVQCGAQGVQVQLVQDDPSGEGVLPSARGPATFLPFLTVDLPVYVLQEVLPSSGGPAGPEATQFPGSTINLQDLQ",
# ATX3_HUMAN
"MESIFHEKQEGSLCAQHCLNNLLQGEYFSPVELSSIAHQLDEEERMRMAEGGVTSEDYRTFLQQPSGNMDDSGFFSIQVISNALKVWGLELILFNSPEYQRLRIDPINERSFICNYKEHWFTVRKLGKQWFNLNSLLTGPELISDTYLALFLAQLQQEGYSIFVVKGDLPDCEADQLLQMIRVQQMHRPKLIGEELAQLKEQRVHKTDLERVLEANDGSGMLDEDEEDLQRALALSRQEIDMEDEEADLRRAIQLSMQGSSRNISQDMTQTSGTNLTSEELRKRREAYFEKQQQKQQQQQQQQQQGDLSGQSSHPCERPATSSGALGSDLGDAMSEEDMLQAAVTMSLETVRNDLKTEGKK",
# ATX3L_HUMAN
"MDFIFHEKQEGFLCAQHCLNNLLQGEYFSPVELASIAHQLDEEERMRMAEGGVTSEEYLAFLQQPSENMDDTGFFSIQVISNALKFWGLEIIHFNNPEYQKLGIDPINERSFICNYKQHWFTIRKFGKHWFNLNSLLAGPELISDTCLANFLARLQQQAYSVFVVKGDLPDCEADQLLQIISVEEMDTPKLNGKKLVKQKEHRVYKTVLEKVSEESDESGTSDQDEEDFQRALELSRQETNREDEHLRSTIELSMQGSSGNTSQDLPKTSCVTPASEQPKKIKEDYFEKHQQEQKQQQQQSDLPGHSSYLHERPTTSSRAIESDLSDDISEGTVQAAVDTILEIMRKNLKIKGEK",
# BAP1_HUMAN
"MNKGWLELESDPGLFTLLVEDFGVKGVQVEEIYDLQSKCQGPVYGFIFLFKWIEERRSRRKVSTLVDDTSVIDDDIVNNMFFAHQLIPNSCATHALLSVLLNCSSVDLGPTLSRMKDFTKGFSPESKGYAIGNAPELAKAHNSHARPEPRHLPEKQNGLSAVRTMEAFHFVSYVPITGRLFELDGLKVYPIDHGPWGEDEEWTDKARRVIMERIGLATAGEPYHDIRFNLMAVVPDRRIKYEARLHVLKVNRQTVLEALQQLIRVTQPELIQTHKSQESQLPEESKSASNKSPLVLEANRAPAASEGNHTDGAEEAAGSCAQAPSHSPPNKPKLVVKPPGSSLNGVHPNPTPIVQRLPAFLDNHNYAKSPMQEEEDLAAGVGRSRVPVRPPQQYSDDEDDYEDDEEDDVQNTNSALRYKGKGTGKPGALSGSADGQLSVLQPNTINVLAEKLKESQKDLSIPLSIKTSSGAGSPAVAVPTHSQPSPTPSNESTDTASEIGSAFNSPLRSPIRSANPTRPSSPVTSHISKVLFGEDDSLLRVDCIRYNRAVRDLGPVISTGLLHLAEDGVLSPLALTEGGKGSSPSIRPIQGSQGSSSPVEKEVVEATDSREKTGMVRPGEPLSGEKYSPKELLALLKCVEAEIANYEACLKEEVEKRKKFKIDDQRRTHNYDEFICTFISMLAQEGMLANLVEQNISVRRRQGVSIGRLHKQRKPDRRKRSRPYKAKRQ",
# BRCC3_HUMAN
"MAVQVVQAVQAVHLESDAFLVCLNHALSTEKEEVMGLCIGELNDDTRSDSKFAYTGTEMRTVAEKVDAVRIVHIHSVIILRRSDKRKDRVEISPEQLSAASTEAERLAELTGRPMRVVGWYHSHPHITVWPSHVDVRTQAMYQMMDQGFVGLIFSCFIEDKNTKTGRVLYTCFQSIQAQKSSESLHGPRDFWSSSQHISIEGQKEEERYERIEIPIHIVPHVTIGKVCLESAVELPKILCQEEQDAYRRIHSLTHLDSVTKIHNGSVFTKNLCSQMSAVSGPLLQWLEDRLEQNQQHLQELQQEKEELMQELSSLE",
# CSN5_HUMAN
"MAASGSGMAQKTWELANNMQEAQSIDEIYKYDKKQQQEILAAKPWTKDHHYFKYCKISALALLKMVMHARSGGNLEVMGLMLGKVDGETMIIMDSFALPVEGTETRVNAQAAAYEYMAAYIENAKQVGRLENAIGWYHSHPGYGCWLSGIDVSTQMLNQQFQEPFVAVVIDPTRTISAGKVNLGAFRTYPKGYKPPDEGPSEYQTIPLNKIEDFGVHCKQYYALEVSYFKSSLDRKLLELLWNKYWVNTLSSSSLLTNADYTTGQVFDLSEKLEQSEAQLGRGSFMLGLETHDRKSEDKLAKATRDSCKTTIEAIHGLMSQVIKDKLFNQINIS",
# CSN6_HUMAN
"MAAAAAAAAATNGTGGSSGMEVDAAVVPSVMACGVTGSVSVALHPLVILNISDHWIRMRSQEGRPVQVIGALIGKQEGRNIEVMNSFELLSHTVEEKIIIDKEYYYTKEEQFKQVFKELEFLGWYTTGGPPDPSDIHVHKQVCEIIESPLFLKLNPMTKHTDLPVSVFESVIDIINGEATMLFAELTYTLATEEAERIGVDHVARMTATGSGENSTVAEHLIAQHSAIKMLHSRVKLILEYVKASEAGEVPFNHEILREAYALCHCLPVLSTDKFKTDFYDQCNDVGLMAYLGTITKTCNTMNQFVNKFNVLYDRQGIGRRMRGLFF",
# CYLD_HUMAN
"MSSGLWSQEKVTSPYWEERIFYLLLQECSVTDKQTQKLLKVPKGSIGQYIQDRSVGHSRIPSAKGKKNQIGLKILEQPHAVLFVDEKDVVEINEKFTELLLAITNCEERFSLFKNRNRLSKGLQIDVGCPVKVQLRSGEEKFPGVVRFRGPLLAERTVSGIFFGVELLEEGRGQGFTDGVYQGKQLFQCDEDCGVFVALDKLELIEDDDTALESDYAGPGDTMQVELPPLEINSRVSLKVGETIESGTVIFCDVLPGKESLGYFVGVDMDNPIGNWDGRFDGVQLCSFACVESTILLHINDIIPALSESVTQERRPPKLAFMSRGVGDKGSSSHNKPKATGSTSDPGNRNRSELFYTLNGSSVDSQPQSKSKNTWYIDEVAEDPAKSLTEISTDFDRSSPPLQPPPVNSLTTENRFHSLPFSLTKMPNTNGSIGHSPLSLSAQSVMEELNTAPVQESPPLAMPPGNSHGLEVGSLAEVKENPPFYGVIRWIGQPPGLNEVLAGLELEDECAGCTDGTFRGTRYFTCALKKALFVKLKSCRPDSRFASLQPVSNQIERCNSLAFGGYLSEVVEENTPPKMEKEGLEIMIGKKKGIQGHYNSCYLDSTLFCLFAFSSVLDTVLLRPKEKNDVEYYSETQELLRTEIVNPLRIYGYVCATKIMKLRKILEKVEAASGFTSEEKDPEEFLNILFHHILRVEPLLKIRSAGQKVQDCYFYQIFMEKNEKVGVPTIQQLLEWSFINSNLKFAEAPSCLIIQMPRFGKDFKLFKKIFPSLELNITDLLEDTPRQCRICGGLAMYECRECYDDPDISAGKIKQFCKTCNTQVHLHPKRLNHKYNPVSLPKDLPDWDWRHGCIPCQNMELFAVLCIETSHYVAFVKYGKDDSAWLFFDSMADRDGGQNGFNIPQVTPCPEVGEYLKMSLEDLHSLDSRRIQGCARRLLCDAYMCMYQSPTMSLYK",
# DESI1_HUMAN
"MEPPNLYPVKLYVYDLSKGLARRLSPIMLGKQLEGIWHTSIVVHKDEFFFGSGGISSCPPGGTLLGPPDSVVDVGSTEVTEEIFLEYLSSLGESLFRGEAYNLFEHNCNTFSNEVAQFLTGRKIPSYITDLPSEVLSTPFGQALRPLLDSIQIQPPGGSSVGRPNGQS",
# DESI2_HUMAN
"MGANQLVVLNVYDMYWMNEYTSSIGIGVFHSGIEVYGREFAYGGHPYPFSGIFEISPGNASELGETFKFKEAVVLGSTDFLEDDIEKIVEELGKEYKGNAYHLMHKNCNHFSSALSEILCGKEIPRWINRLAYFSSCIPFLQSCLPKEWLTPAALQSSVSQELQDELEEAEDAAASASVASTAAGSRPGRHTKL",
# EIF3F_HUMAN
"MATPAVPVSAPPATPTPVPAAAPASVPAPTPAPAAAPVPAAAPASSSDPAAAAAATAAPGQTPASAQAPAQTPAPALPGPALPGPFPGGRVVRLHPVILASIVDSYERRNEGAARVIGTLLGTVDKHSVEVTNCFSVPHNESEDEVAVDMEFAKNMYELHKKVSPNELILGWYATGHDITEHSVLIHEYYSREAPNPIHLTVDTSLQNGRMSIKAYVSTLMGVPGRTMGVMFTPLTVKYAYYDTERIGVDLIMKTCFSPNRVIGLSSDLQQVGGASARIQDALSTVLQYAEDVLSGKVSADNTVGRFLMSLVNQVPKIVPDDFETMLNSNINDLLMVTYLANLTQSQIALNEKLVNL",
# EIF3H_HUMAN
"MASRKEGTGSTATSSSSTAGAAGKGKGKGGSGDSAVKQVQIDGLVVLKIIKHYQEEGQGTEVVQGVLLGLVVEDRLEITNCFPFPQHTEDDADFDEVQYQMEMMRSLRHVNIDHLHVGWYQSTYYGSFVTRALLDSQFSYQHAIEESVVLIYDPIKTAQGSLSLKAYRLTPKLMEVCKEKDFSPEALKKANITFEYMFEEVPIVIKNSHLINVLMWELEKKSAVADKHELLSLASSNHLGKNLQLLMDRVDEMSQDIVKYNTYMRNTSKQQQQKHQYQQRRQQENMQRQSRGEPPLPEEDLSKLFKPPQPPARMDSLLIAGQINTYCQNIKEFTAQNLGKLFMAQALQEYNN",
# JOS1_HUMAN
"MSCVPWKGDKAKSESLELPQAAPPQIYHEKQRRELCALHALNNVFQDSNAFTRDTLQEIFQRLSPNTMVTPHKKSMLGNGNYDVNVIMAALQTKGYEAVWWDKRRDVGVIALTNVMGFIMNLPSSLCWGPLKLPLKRQHWICVREVGGAYYNLDSKLKMPEWIGGESELRKFLKHHLRGKNCELLLVVPEEVEAHQSWRTDV",
# JOS2_HUMAN
"MSQAPGAQPSPPTVYHERQRLELCAVHALNNVLQQQLFSQEAADEICKRLAPDSRLNPHRSLLGTGNYDVNVIMAALQGLGLAAVWWDRRRPLSQLALPQVLGLILNLPSPVSLGLLSLPLRRRHWVALRQVDGVYYNLDSKLRAPEALGDEDGVRAFLAAALAQGLCEVLLVVTKEVEEKGSWLRTD",
# MPND_HUMAN
"MAAPEPLSPAGGAGEEAPEEDEDEAEAEDPERPNAGAGGGRSGGGGSSVSGGGGGGGAGAGGCGGPGGALTRRAVTLRVLLKDALLEPGAGVLSIYYLGKKFLGDLQPDGRIMWQETGQTFNSPSAWATHCKKLVNPAKKSGCGWASVKYKGQKLDKYKATWLRLHQLHTPATAADESPASEGEEEELLMEEEEEDVLAGVSAEDKSRRPLGKSPSEPAHPEATTPGKRVDSKIRVPVRYCMLGSRDLARNPHTLVEVTSFAAINKFQPFNVAVSSNVLFLLDFHSHLTRSEVVGYLGGRWDVNSQMLTVLRAFPCRSRLGDAETAAAIEEEIYQSLFLRGLSLVGWYHSHPHSPALPSLQDIDAQMDYQLRLQGSSNGFQPCLALLCSPYYSGNPGPESKISPFWVMPPPEMLLVEFYKGSPDLVRLQEPWSQEHTYLDKLKISLASRTPKDQSLCHVLEQVCGVLKQGS",
# MYSM1_HUMAN
"MAAEEADVDIEGDVVAAAGAQPGSGENTASVLQKDHYLDSSWRTENGLIPWTLDNTISEENRAVIEKMLLEEEYYLSKKSQPEKVWLDQKEDDKKYMKSLQKTAKIMVHSPTKPASYSVKWTIEEKELFEQGLAKFGRRWTKISKLIGSRTVLQVKSYARQYFKNKVKCGLDKETPNQKTGHNLQVKNEDKGTKAWTPSCLRGRADPNLNAVKIEKLSDDEEVDITDEVDELSSQTPQKNSSSDLLLDFPNSKMHETNQGEFITSDSQEALFSKSSRGCLQNEKQDETLSSSEITLWTEKQSNGDKKSIELNDQKFNELIKNCNKHDGRGIIVDARQLPSPEPCEIQKNLNDNEMLFHSCQMVEESHEEEELKPPEQEIEIDRNIIQEEEKQAIPEFFEGRQAKTPERYLKIRNYILDQWEICKPKYLNKTSVRPGLKNCGDVNCIGRIHTYLELIGAINFGCEQAVYNRPQTVDKVRIRDRKDAVEAYQLAQRLQSMRTRRRRVRDPWGNWCDAKDLEGQTFEHLSAEELAKRREEEKGRPVKSLKVPRPTKSSFDPFQLIPCNFFSEEKQEPFQVKVASEALLIMDLHAHVSMAEVIGLLGGRYSEVDKVVEVCAAEPCNSLSTGLQCEMDPVSQTQASETLAVRGFSVIGWYHSHPAFDPNPSLRDIDTQAKYQSYFSRGGAKFIGMIVSPYNRNNPLPYSQITCLVISEEISPDGSYRLPYKFEVQQMLEEPQWGLVFEKTRWIIEKYRLSHSSVPMDKIFRRDSDLTCLQKLLECMRKTLSKVTNCFMAEEFLTEIENLFLSNYKSNQENGVTEENCTKELLM",
# OTUB1_HUMAN
"MAAEEPQQQKQEPLGSDSEGVNCLAYDEAIMAQQDRIQQEIAVQNPLVSERLELSVLYKEYAEDDNIYQQKIKDLHKKYSYIRKTRPDGNCFYRAFGFSHLEALLDDSKELQRFKAVSAKSKEDLVSQGFTEFTIEDFHNTFMDLIEQVEKQTSVADLLASFNDQSTSDYLVVYLRLLTSGYLQRESKFFEHFIEGGRTVKEFCQQEVEPMCKESDHIHIIALAQALSVSIQVEYMDRGEGGTTNPHIFPEGSEPKVYLLYRPGHYDILYK",
# OTUB2_HUMAN
"MSETSFNLISEKCDILSILRDHPENRIYRRKIEELSKRFTAIRKTKGDGNCFYRALGYSYLESLLGKSREIFKFKERVLQTPNDLLAAGFEEHKFRNFFNAFYSVVELVEKDGSVSSLLKVFNDQSASDHIVQFLRLLTSAFIRNRADFFRHFIDEEMDIKDFCTHEVEPMATECDHIQITALSQALSIALQVEYVDEMDTALNHHVFPEAATPSVYLLYKTSHYNILYAADKH",
# OTUD1_HUMAN
"MQLYSSVCTHYPAGAPGPTAAAPAPPAAATPFKVSLQPPGAAGAAPEPETGECQPAAAAEHREAAAVPAAKMPAFSSCFEVVSGAAAPASAAAGPPGASCKPPLPPHYTSTAQITVRALGADRLLLHGPDPVPGAAGSAAAPRGRCLLLAPAPAAPVPPRRGSSAWLLEELLRPDCPEPAGLDATREGPDRNFRLSEHRQALAAAKHRGPAATPGSPDPGPGPWGEEHLAERGPRGWERGGDRCDAPGGDAARRPDPEAEAPPAGSIEAAPSSAAEPVIVSRSDPRDEKLALYLAEVEKQDKYLRQRNKYRFHIIPDGNCLYRAVSKTVYGDQSLHRELREQTVHYIADHLDHFSPLIEGDVGEFIIAAAQDGAWAGYPELLAMGQMLNVNIHLTTGGRLESPTVSTMIHYLGPEDSLRPSIWLSWLSNGHYDAVFDHSYPNPEYDNWCKQTQVQRKRDEELAKSMAISLSKMYIEQNACS",
# OTUD3_HUMAN
"MSRKQAAKSRPGSGSRKAEAERKRDERAARRALAKERRNRPESGGGGGCEEEFVSFANQLQALGLKLREVPGDGNCLFRALGDQLEGHSRNHLKHRQETVDYMIKQREDFEPFVEDDIPFEKHVASLAKPGTFAGNDAIVAFARNHQLNVVIHQLNAPLWQIRGTEKSSVRELHIAYRYGEHYDSVRRINDNSEAPAHLQTDFQMLHQDESNKREKIKTKGMDSEDDLRDEVEDAVQKVCNATGCSDFNLIVQNLEAENYNIESAIIAVLRMNQGKRNNAEENLEPSGRVLKQCGPLWEEGGSGARIFGNQGLNEGRTENNKAQASPSEENKANKNQLAKVTNKQRREQQWMEKKKRQEERHRHKALESRGSHRDNNRSEAEANTQVTLVKTFAALNI",
# OTUD4_HUMAN
"MEAAVGVPDGGDQGGAGPREDATPMDAYLRKLGLYRKLVAKDGSCLFRAVAEQVLHSQSRHVEVRMACIHYLRENREKFEAFIEGSFEEYLKRLENPQEWVGQVEISALSLMYRKDFIIYREPNVSPSQVTENNFPEKVLLCFSNGNHYDIVYPIKYKESSAMCQSLLYELLYEKVFKTDVSKIVMELDTLEVADEDNSEISDSEDDSCKSKTAAAAADVNGFKPLSGNEQLKNNGNSTSLPLSRKVLKSLNPAVYRNVEYEIWLKSKQAQQKRDYSIAAGLQYEVGDKCQVRLDHNGKFLNADVQGIHSENGPVLVEELGKKHTSKNLKAPPPESWNTVSGKKMKKPSTSGQNFHSDVDYRGPKNPSKPIKAPSALPPRLQHPSGVRQHAFSSHSSGSQSQKFSSEHKNLSRTPSQIIRKPDRERVEDFDHTSRESNYFGLSPEERREKQAIEESRLLYEIQNRDEQAFPALSSSSVNQSASQSSNPCVQRKSSHVGDRKGSRRRMDTEERKDKDSIHGHSQLDKRPEPSTLENITDDKYATVSSPSKSKKLECPSPAEQKPAEHVSLSNPAPLLVSPEVHLTPAVPSLPATVPAWPSEPTTFGPTGVPAPIPVLSVTQTLTTGPDSAVSQAHLTPSPVPVSIQAVNQPLMPLPQTLSLYQDPLYPGFPCNEKGDRAIVPPYSLCQTGEDLPKDKNILRFFFNLGVKAYSCPMWAPHSYLYPLHQAYLAACRMYPKVPVPVYPHNPWFQEAPAAQNESDCTCTDAHFPMQTEASVNGQMPQPEIGPPTFSSPLVIPPSQVSESHGQLSYQADLESETPGQLLHADYEESLSGKNMFPQPSFGPNPFLGPVPIAPPFFPHVWYGYPFQGFIENPVMRQNIVLPSDEKGELDLSLENLDLSKDCGSVSTVDEFPEARGEHVHSLPEASVSSKPDEGRTEQSSQTRKADTALASIPPVAEGKAHPPTQILNRERETVPVELEPKRTIQSLKEKTEKVKDPKTAADVVSPGANSVDSRVQRPKEESSEDENEVSNILRSGRSKQFYNQTYGSRKYKSDWGYSGRGGYQHVRSEESWKGQPSRSRDEGYQYHRNVRGRPFRGDRRRSGMGDGHRGQHT",
# OTUD5_HUMAN
"MTILPKKKPPPPDADPANEPPPPGPMPPAPRRGGGVGVGGGGTGVGGGDRDRDSGVVGARPRASPPPQGPLPGPPGALHRWALAVPPGAVAGPRPQQASPPPCGGPGGPGGGPGDALGAAAAGVGAAGVVVGVGGAVGVGGCCSGPGHSKRRRQAPGVGAVGGGSPEREEVGAGYNSEDEYEAAAARIEAMDPATVEQQEHWFEKALRDKKGFIIKQMKEDGACLFRAVADQVYGDQDMHEVVRKHCMDYLMKNADYFSNYVTEDFTTYINRKRKNNCHGNHIEMQAMAEMYNRPVEVYQYSTGTSAVEPINTFHGIHQNEDEPIRVSYHRNIHYNSVVNPNKATIGVGLGLPSFKPGFAEQSLMKNAIKTSEESWIEQQMLEDKKRATDWEATNEAIEEQVARESYLQWLRDQEKQARQVRGPSQPRKASATCSSATAAASSGLEEWTSRSPRQRSSASSPEHPELHAELGMKPPSPGTVLALAKPPSPCAPGTSSQFSAGADRATSPLVSLYPALECRALIQQMSPSAFGLNDWDDDEILASVLAVSQQEYLDSMKKNKVHRDPPPDKS",
# OTU6A_HUMAN
"MDDPKSEQQRILRRHQRERQELQAQIRSLKNSVPKTDKTKRKQLLQDVARMEAEMAQKHRQELEKFQDDSSIESVVEDLAKMNLENRPPRSSKAHRKRERMESEERERQESIFQAEMSEHLAGFKREEEEKLAAILGARGLEMKAIPADGHCMYRAIQDQLVFSVSVEMLRCRTASYMKKHVDEFLPFFSNPETSDSFGYDDFMIYCDNIVRTTAWGGQLELRALSHVLKTPIEVIQADSPTLIIGEEYVKKPIILVYLRYAYSLGEHYNSVTPLEAGAAGGVLPRLL",
# OTU6B_HUMAN
"MEAVLTEELDEEEQLLRRHRKEKKELQAKIQGMKNAVPKNDKKRRKQLTEDVAKLEKEMEQKHREELEQLKLTTKENKIDSVAVNISNLVLENQPPRISKAQKRREKKAALEKEREERIAEAEIENLTGARHMESEKLAQILAARQLEIKQIPSDGHCMYKAIEDQLKEKDCALTVVALRSQTAEYMQSHVEDFLPFLTNPNTGDMYTPEEFQKYCEDIVNTAAWGGQLELRALSHILQTPIEIIQADSPPIIVGEEYSKKPLILVYMRHAYGLGEHYNSVTRLVNIVTENCS",
# OTU7A_HUMAN
"MVSSVLPNPTSAECWAALLHDPMTLDMDAVLSDFVRSTGAEPGLARDLLEGKNWDLTAALSDYEQLRQVHTANLPHVFNEGRGPKQPEREPQPGHKVERPCLQRQDDIAQEKRLSRGISHASSAIVSLARSHVASECNNEQFPLEMPIYTFQLPDLSVYSEDFRSFIERDLIEQATMVALEQAGRLNWWSTVCTSCKRLLPLATTGDGNCLLHAASLGMWGFHDRDLVLRKALYTMMRTGAEREALKRRWRWQQTQQNKEEEWEREWTELLKLASSEPRTHFSKNGGTGGGVDNSEDPVYESLEEFHVFVLAHILRRPIVVVADTMLRDSGGEAFAPIPFGGIYLPLEVPPNRCHCSPLVLAYDQAHFSALVSMEQRDQQREQAVIPLTDSEHKLLPLHFAVDPGKDWEWGKDDNDNARLAHLILSLEAKLNLLHSYMNVTWIRIPSETRAPLAQPESPTASAGEDVQSLADSLDSDRDSVCSNSNSNNGKNGKDKEKEKQRKEKDKTRADSVANKLGSFSKTLGIKLKKNMGGLGGLVHGKMGRANSANGKNGDSAERGKEKKAKSRKGSKEESGASASTSPSEKTTPSPTDKAAGASPAEKGGGPRGDAWKYSTDVKLSLNILRAAMQGERKFIFAGLLLTSHRHQFHEEMIGYYLTSAQERFSAEQEQRRRDAATAAAAAAAAAAATAKRPPRRPETEGVPVPERASPGPPTQLVLKLKERPSPGPAAGRAARAAAGGTASPGGGARRASASGPVPGRSPPAPARQSVIHVQASGARDEACAPAVGALRPCATYPQQNRSLSSQSYSPARAAALRTVNTVESLARAVPGALPGAAGTAGAAEHKSQTYTNGFGALRDGLEFADADAPTARSNGECGRGGPGPVQRRCQRENCAFYGRAETEHYCSYCYREELRRRREARGARP",
# OTU7B_HUMAN
"MTLDMDAVLSDFVRSTGAEPGLARDLLEGKNWDVNAALSDFEQLRQVHAGNLPPSFSEGSGGSRTPEKGFSDREPTRPPRPILQRQDDIVQEKRLSRGISHASSSIVSLARSHVSSNGGGGGSNEHPLEMPICAFQLPDLTVYNEDFRSFIERDLIEQSMLVALEQAGRLNWWVSVDPTSQRLLPLATTGDGNCLLHAASLGMWGFHDRDLMLRKALYALMEKGVEKEALKRRWRWQQTQQNKESGLVYTEDEWQKEWNELIKLASSEPRMHLGTNGANCGGVESSEEPVYESLEEFHVFVLAHVLRRPIVVVADTMLRDSGGEAFAPIPFGGIYLPLEVPASQCHRSPLVLAYDQAHFSALVSMEQKENTKEQAVIPLTDSEYKLLPLHFAVDPGKGWEWGKDDSDNVRLASVILSLEVKLHLLHSYMNVKWIPLSSDAQAPLAQPESPTASAGDEPRSTPESGDSDKESVGSSSTSNEGGRRKEKSKRDREKDKKRADSVANKLGSFGKTLGSKLKKNMGGLMHSKGSKPGGVGTGLGGSSGTETLEKKKKNSLKSWKGGKEEAAGDGPVSEKPPAESVGNGGSKYSQEVMQSLSILRTAMQGEGKFIFVGTLKMGHRHQYQEEMIQRYLSDAEERFLAEQKQKEAERKIMNGGIGGGPPPAKKPEPDAREEQPTGPPAESRAMAFSTGYPGDFTIPRPSGGGVHCQEPRRQLAGGPCVGGLPPYATFPRQCPPGRPYPHQDSIPSLEPGSHSKDGLHRGALLPPPYRVADSYSNGYREPPEPDGWAGGLRGLPPTQTKCKQPNCSFYGHPETNNFCSCCYREELRRREREPDGELLVHRF",
# OTUL_HUMAN
"MSRGTMPQPEAWPGASCAETPAREAAATARDGGKAAASGQPRPEMQCPAEHEEDMYRAADEIEKEKELLIHERGASEPRLSVAPEMDIMDYCKKEWRGNTQKATCMKMGYEEVSQKFTSIRRVRGDNYCALRATLFQAMSQAVGLPPWLQDPELMLLPEKLISKYNWIKQWKLGLKFDGKNEDLVDKIKESLTLLRKKWAGLAEMRTAEARQIACDELFTNEAEEYSLYEAVKFLMLNRAIELYNDKEKGKEVPFFSVLLFARDTSNDPGQLLRNHLNQVGHTGGLEQVEMFLLAYAVRHTIQVYRLSKYNTEEFITVYPTDPPKDWPVVTLIAEDDRHYNIPVRVCEETSL",
# PAN2_HUMAN
"MNFEGLDPGLAEYAPAMHSALDPVLDAHLNPSLLQNVELDPEGVALEALPVQESVHIMEGVYSELHSVVAEVGVPVSVSHFDLHEEMLWVGSHGGHATSFFGPALERYSSFQVNGSDDIRQIQSLENGILFLTKNNLKYMARGGLIIFDYLLDENEDMHSLLLTDSSTLLVGGLQNHIIEIDLNTVQETQKYAVETPGVTIMRQTNRFFFCGHTSGKVSLRDLRTFKVEHEFDAFSGSLSDFDVHGNLLAACGFSSRLTGLACDRFLKVYDLRMMRAITPLQVHVDPAFLRFIPTYTSRLAIISQSGQCQFCEPTGLANPADIFHVNPVGPLLMTFDVSASKQALAFGDSEGCVHLWTDSPEPSFNPYSRETEFALPCLVDSLPPLDWSQDLLPLSLIPVPLTTDTLLSDWPAANSAPAPRRAPPVDAEILRTMKKVGFIGYAPNPRTRLRNQIPYRLKESDSEFDSFSQVTESPVGREEEPHLHMVSKKYRKVTIKYSKLGLEDFDFKHYNKTLFAGLEPHIPNAYCNCMIQVLYFLEPVRCLIQNHLCQKEFCLACELGFLFHMLDLSRGDPCQGNNFLRAFRTIPEASALGLILADSDEASGKGNLARLIQRWNRFILTQLHQDMQELEIPQAYRGAGGSSFCSSGDSVIGQLFSCEMENCSLCRCGSETVRASSTLLFTLSYPDGSKSDKTGKNYDFAQVLKRSICLDQNTQAWCDTCEKYQPTIQTRNIRHLPDILVINCEVNSSKEADFWRMQAEVAFKMAVKKHGGEISKNKEFALADWKELGSPEGVLVCPSIEELKNVWLPFSIRMKMTKNKGLDVCNWTDGDEMQWGPARAEEEHGVYVYDLMATVVHILDSRTGGSLVAHIKVGETYHQRKEGVTHQQWYLFNDFLIEPIDKHEAVQFDMNWKVPAILYYVKRNLNSRYNLNIKNPIEASVLLAEASLARKQRKTHTTFIPLMLNEMPQIGDLVGLDAEFVTLNEEEAELRSDGTKSTIKPSQMSVARITCVRGQGPNEGIPFIDDYISTQEQVVDYLTQYSGIKPGDLDAKISSKHLTTLKSTYLKLRFLIDIGVKFVGHGLQKDFRVINLMVPKDQVLDTVYLFHMPRKRMISLRFLAWYFLDLKIQGETHDSIEDARTALQLYRKYLELSKNGTEPESFHKVLKGLYEKGRKMDWKVPEPEGQTSPKNAAVFSSVLAL",
# PAR11_HUMAN
"MWEANPEMFHKAEELFSKTTNNEVDDMDTSDTQWGWFYLAECGKWHMFQPDTNSQCSVSSEDIEKSFKTNPCGSISFTTSKFSYKIDFAEMKQMNLTTGKQRLIKRAPFSISAFSYICENEAIPMPPHWENVNTQVPYQLIPLHNQTHEYNEVANLFGKTMDRNRIKRIQRIQNLDLWEFFCRKKAQLKKKRGVPQINEQMLFHGTSSEFVEAICIHNFDWRINGIHGAVFGKGTYFARDAAYSSRFCKDDIKHGNTFQIHGVSLQQRHLFRTYKSMFLARVLIGDYINGDSKYMRPPSKDGSYVNLYDSCVDDTWNPKIFVVFDANQIYPEYLIDFH",
# PRP8_HUMAN
"MAGVFPYRGPGNPVPGPLAPLPDYMSEEKLQEKARKWQQLQAKRYAEKRKFGFVDAQKEDMPPEHVRKIIRDHGDMTNRKFRHDKRVYLGALKYMPHAVLKLLENMPMPWEQIRDVPVLYHITGAISFVNEIPWVIEPVYISQWGSMWIMMRREKRDRRHFKRMRFPPFDDEEPPLDYADNILDVEPLEAIQLELDPEEDAPVLDWFYDHQPLRDSRKYVNGSTYQRWQFTLPMMSTLYRLANQLLTDLVDDNYFYLFDLKAFFTSKALNMAIPGGPKFEPLVRDINLQDEDWNEFNDINKIIIRQPIRTEYKIAFPYLYNNLPHHVHLTWYHTPNVVFIKTEDPDLPAFYFDPLINPISHRHSVKSQEPLPDDDEEFELPEFVEPFLKDTPLYTDNTANGIALLWAPRPFNLRSGRTRRALDIPLVKNWYREHCPAGQPVKVRVSYQKLLKYYVLNALKHRPPKAQKKRYLFRSFKATKFFQSTKLDWVEVGLQVCRQGYNMLNLLIHRKNLNYLHLDYNFNLKPVKTLTTKERKKSRFGNAFHLCREVLRLTKLVVDSHVQYRLGNVDAFQLADGLQYIFAHVGQLTGMYRYKYKLMRQIRMCKDLKHLIYYRFNTGPVGKGPGCGFWAAGWRVWLFFMRGITPLLERWLGNLLARQFEGRHSKGVAKTVTKQRVESHFDLELRAAVMHDILDMMPEGIKQNKARTILQHLSEAWRCWKANIPWKVPGLPTPIENMILRYVKAKADWWTNTAHYNRERIRRGATVDKTVCKKNLGRLTRLYLKAEQERQHNYLKDGPYITAEEAVAVYTTTVHWLESRRFSPIPFPPLSYKHDTKLLILALERLKEAYSVKSRLNQSQREELGLIEQAYDNPHEALSRIKRHLLTQRAFKEVGIEFMDLYSHLVPVYDVEPLEKITDAYLDQYLWYEADKRRLFPPWIKPADTEPPPLLVYKWCQGINNLQDVWETSEGECNVMLESRFEKMYEKIDLTLLNRLLRLIVDHNIADYMTAKNNVVINYKDMNHTNSYGIIRGLQFASFIVQYYGLVMDLLVLGLHRASEMAGPPQMPNDFLSFQDIATEAAHPIRLFCRYIDRIHIFFRFTADEARDLIQRYLTEHPDPNNENIVGYNNKKCWPRDARMRLMKHDVNLGRAVFWDIKNRLPRSVTTVQWENSFVSVYSKDNPNLLFNMCGFECRILPKCRTSYEEFTHKDGVWNLQNEVTKERTAQCFLRVDDESMQRFHNRVRQILMASGSTTFTKIVNKWNTALIGLMTYFREAVVNTQELLDLLVKCENKIQTRIKIGLNSKMPSRFPPVVFYTPKELGGLGMLSMGHVLIPQSDLRWSKQTDVGITHFRSGMSHEEDQLIPNLYRYIQPWESEFIDSQRVWAEYALKRQEAIAQNRRLTLEDLEDSWDRGIPRINTLFQKDRHTLAYDKGWRVRTDFKQYQVLKQNPFWWTHQRHDGKLWNLNNYRTDMIQALGGVEGILEHTLFKGTYFPTWEGLFWEKASGFEESMKWKKLTNAQRSGLNQIPNRRFTLWWSPTINRANVYVGFQVQLDLTGIFMHGKIPTLKISLIQIFRAHLWQKIHESIVMDLCQVFDQELDALEIETVQKETIHPRKSYKMNSSCADILLFASYKWNVSRPSLLADSKDVMDSTTTQKYWIDIQLRWGDYDSHDIERYARAKFLDYTTDNMSIYPSPTGVLIAIDLAYNLHSAYGNWFPGSKPLIQQAMAKIMKANPALYVLRERIRKGLQLYSSEPTEPYLSSQNYGELFSNQIIWFVDDTNVYRVTIHKTFEGNLTTKPINGAIFIFNPRTGQLFLKIIHTSVWAGQKRLGQLAKWKTAEEVAALIRSLPVEEQPKQIIVTRKGMLDPLEVHLLDFPNIVIKGSELQLPFQACLKVEKFGDLILKATEPQMVLFNLYDDWLKTISSYTAFSRLILILRALHVNNDRAKVILKPDKTTITEPHHIWPTLTDEEWIKVEVQLKDLILADYGKKNNVNVASLTQSEIRDIILGMEISAPSQQRQQIAEIEKQTKEQSQLTATQTRTVNKHGDEIITSTTSNYETQTFSSKTEWRVRAISAANLHLRTNHIYVSSDDIKETGYTYILPKNVLKKFICISDLRAQIAGYLYGVSPPDNPQVKEIRCIVMVPQWGTHQTVHLPGQLPQHEYLKEMEPLGWIHTQPNESPQLSPQDVTTHAKIMADNPSWDGEKTIIITCSFTPGSCTLTAYKLTPSGYEWGRQNTDKGNNPKGYLPSHYERVQMLLSDRFLGFFMVPAQSSWNYNFMGVRHDPNMKYELQLANPKEFYHEVHRPSHFLNFALLQEGEVYSADREDLYA",
# PSDE_HUMAN
"MDRLLRLGGGMPGLGQGPPTDAPAVDTAEQVYISSLALLKMLKHGRAGVPMEVMGLMLGEFVDDYTVRVIDVFAMPQSGTGVSVEAVDPVFQAKMLDMLKQTGRPEMVVGWYHSHPGFGCWLSGVDINTQQSFEALSERAVAVVVDPIQSVKGKVVIDAFRLINANMMVLGHEPRQTTSNLGHLNKPSIQALIHGLNRHYYSITINYRKNELEQKMLLNLHKKSWMEGLTLQDYSEHCKHNESVVKEMLELAKNYNKAVEEEDKMTPEQLAIKNVGKQDPKRHLEEHVDVLMTSNIVQCLAAMLDTVVFK",
# PSMD7_HUMAN
"MPELAVQKVVVHPLVLLSVVDHFNRIGKVGNQKRVVGVLLGSWQKKVLDVSNSFAVPFDEDDKDDSVWFLDHDYLENMYGMFKKVNARERIVGWYHTGPKLHKNDIAINELMKRYCPNSVLVIIDVKPKDLGLPTEAYISVEEVHDDGTPTSKTFEHVTSEIGAEEAEEVGVEHLLRDIKDTTVGTLSQRITNQVHGLKGLNSKLLDIRSYLEKVATGKLPINHQIIYQLQDVFNLLPDVSLQEFVKAFYLKTNDQMVVVYLASLIRSVVALHNLINNKIANRDAEKKEGQEKEESKKDRKEDKEKDKDKEKSDVKKEEKKEKK",
# SENP1_HUMAN
"MDDIADRMRMDAGEVTLVNHNSVFKTHLLPQTGFPEDQLSLSDQQILSSRQGHLDRSFTCSTRSAAYNPSYYSDNPSSDSFLGSGDLRTFGQSANGQWRNSTPSSSSSLQKSRNSRSLYLETRKTSSGLSNSFAGKSNHHCHVSAYEKSFPIKPVPSPSWSGSCRRSLLSPKKTQRRHVSTAEETVQEEEREIYRQLLQMVTGKQFTIAKPTTHFPLHLSRCLSSSKNTLKDSLFKNGNSCASQIIGSDTSSSGSASILTNQEQLSHSVYSLSSYTPDVAFGSKDSGTLHHPHHHHSVPHQPDNLAASNTQSEGSDSVILLKVKDSQTPTPSSTFFQAELWIKELTSVYDSRARERLRQIEEQKALALQLQNQRLQEREHSVHDSVELHLRVPLEKEIPVTVVQETQKKGHKLTDSEDEFPEITEEMEKEIKNVFRNGNQDEVLSEAFRLTITRKDIQTLNHLNWLNDEIINFYMNMLMERSKEKGLPSVHAFNTFFFTKLKTAGYQAVKRWTKKVDVFSVDILLVPIHLGVHWCLAVVDFRKKNITYYDSMGGINNEACRILLQYLKQESIDKKRKEFDTNGWQLFSKKSQEIPQQMNGSDCGMFACKYADCITKDRPINFTQQHMPYFRKRMVWEILHRKLL",
# SENP2_HUMAN
"MYRWLVRILGTIFRFCDRSVPPARALLKRRRSDSTLFSTVDTDEIPAKRPRLDCFIHQVKNSLYNAASLFGFPFQLTTKPMVTSACNGTRNVAPSGEVFSNSSSCELTGSGSWNNMLKLGNKSPNGISDYPKIRVTVTRDQPRRVLPSFGFTLNSEGCNRRPGGRRHSKGNPESSLMWKPQEQAVTEMISEESGKGLRRPHCTVEEGVQKEEREKYRKLLERLKESGHGNSVCPVTSNYHSSQRSQMDTLKTKGWGEEQNHGVKTTQFVPKQYRLVETRGPLCSLRSEKRCSKGKITDTETMVGIRFENESRRGYQLEPDLSEEVSARLRLGSGSNGLLRRKVSIIETKEKNCSGKERDRRTDDLLELTEDMEKEISNALGHGPQDEILSSAFKLRITRGDIQTLKNYHWLNDEVINFYMNLLVERNKKQGYPALHVFSTFFYPKLKSGGYQAVKRWTKGVNLFEQEIILVPIHRKVHWSLVVIDLRKKCLKYLDSMGQKGHRICEILLQYLQDESKTKRNSDLNLLEWTHHSMKPHEIPQQLNGSDCGMFTCKYADYISRDKPITFTQHQMPLFRKKMVWEILHQQLL",
# SENP3_HUMAN
"MKETIQGTGSWGPEPPGPGIPPAYSSPRRERLRWPPPPKPRLKSGGGFGPDPGSGTTVPARRLPVPRPSFDASASEEEEEEEEEEDEDEEEEVAAWRLPPRWSQLGTSQRPRPSRPTHRKTCSQRRRRAMRAFRMLLYSKSTSLTFHWKLWGRHRGRRRGLAHPKNHLSPQQGGATPQVPSPCCRFDSPRGPPPPRLGLLGALMAEDGVRGSPPVPSGPPMEEDGLRWTPKSPLDPDSGLLSCTLPNGFGGQSGPEGERSLAPPDASILISNVCSIGDHVAQELFQGSDLGMAEEAERPGEKAGQHSPLREEHVTCVQSILDEFLQTYGSLIPLSTDEVVEKLEDIFQQEFSTPSRKGLVLQLIQSYQRMPGNAMVRGFRVAYKRHVLTMDDLGTLYGQNWLNDQVMNMYGDLVMDTVPEKVHFFNSFFYDKLRTKGYDGVKRWTKNVDIFNKELLLIPIHLEVHWSLISVDVRRRTITYFDSQRTLNRRCPKHIAKYLQAEAVKKDRLDFHQGWKGYFKMNVARQNNDSDCGAFVLQYCKHLALSQPFSFTQQDMPKLRRQIYKELCHCKLTV",
# SENP5_HUMAN
"MKKQRKILWRKGIHLAFSEKWNTGFGGFKKFYFHQHLCILKAKLGRPVTWNRQLRHFQGRKKALQIQKTWIKDEPLCAKTKFNVATQNVSTLSSKVKRKDAKHFISSSKTLLRLQAEKLLSSAKNSDHEYCREKNLLKAVTDFPSNSALGQANGHRPRTDPQPSDFPMKFNGESQSPGESGTIVVTLNNHKRKGFCYGCCQGPEHHRNGGPLIPKKFQLNQHRRIKLSPLMMYEKLSMIRFRYRILRSQHFRTKSKVCKLRKAQRSWVQKVTGDHQETRRENGEGGSCSPFPSPEPKDPSCRHQPYFPDMDSSAVVKGTNSHVPDCHTKGSSFLGKELSLDEAFPDQQNGSATNAWDQSSCSSPKWECTELIHDIPLPEHRSNTMFISETEREIMTLGQENQTSSVSDDRVKLSVSGADTSVSSVDGPVSQKAVQNENSYQMEEDGSLKQSILSSELLDHPYCKSPLEAPLVCSGLKLENQVGGGKNSQKASPVDDEQLSVCLSGFLDEVMKKYGSLVPLSEKEVLGRLKDVFNEDFSNRKPFINREITNYRARHQKCNFRIFYNKHMLDMDDLATLDGQNWLNDQVINMYGELIMDAVPDKVHFFNSFFHRQLVTKGYNGVKRWTKKVDLFKKSLLLIPIHLEVHWSLITVTLSNRIISFYDSQGIHFKFCVENIRKYLLTEAREKNRPEFLQGWQTAVTKCIPQQKNDSDCGVFVLQYCKCLALEQPFQFSQEDMPRVRKRIYKELCECRLMD",
# SENP6_HUMAN
"MAAGKSGGSAGEITFLEALARSESKRDGGFKNNWSFDHEEESEGDTDKDGTNLLSVDEDEDSETSKGKKLNRRSEIVANSSGEFILKTYVRRNKSESFKTLKGNPIGLNMLSNNKKLSENTQNTSLCSGTVVHGRRFHHAHAQIPVVKTAAQSSLDRKERKEYPPHVQKVEINPVRLSRLQGVERIMKKTEESESQVEPEIKRKVQQKRHCSTYQPTPPLSPASKKCLTHLEDLQRNCRQAITLNESTGPLLRTSIHQNSGGQKSQNTGLTTKKFYGNNVEKVPIDIIVNCDDSKHTYLQTNGKVILPGAKIPKITNLKERKTSLSDLNDPIILSSDDDDDNDRTNRRESISPQPADSACSSPAPSTGKVEAALNENTCRAERELRSIPEDSELNTVTLPRKARMKDQFGNSIINTPLKRRKVFSQEPPDALALSCQSSFDSVILNCRSIRVGTLFRLLIEPVIFCLDFIKIQLDEPDHDPVEIILNTSDLTKCEWCNVRKLPVVFLQAIPAVYQKLSIQLQMNKEDKVWNDCKGVNKLTNLEEQYIILIFQNGLDPPANMVFESIINEIGIKNNISNFFAKIPFEEANGRLVACTRTYEESIKGSCGQKENKIKTVSFESKIQLRSKQEFQFFDEEEETGENHTIFIGPVEKLIVYPPPPAKGGISVTNEDLHCLNEGEFLNDVIIDFYLKYLVLEKLKKEDADRIHIFSSFFYKRLNQRERRNHETTNLSIQQKRHGRVKTWTRHVDIFEKDFIFVPLNEAAHWFLAVVCFPGLEKPKYEPNPHYHENAVIQKCSTVEDSCISSSASEMESCSQNSSAKPVIKKMLNKKHCIAVIDSNPGQEESDPRYKRNICSVKYSVKKINHTASENEEFNKGESTSQKVADRTKSENGLQNESLSSTHHTDGLSKIRLNYSDESPEAGKMLEDELVDFSEDQDNQDDSSDDGFLADDNCSSEIGQWHLKPTICKQPCILLMDSLRGPSRSNVVKILREYLEVEWEVKKGSKRSFSKDVMKGSNPKVPQQNNFSDCGVYVLQYVESFFENPILSFELPMNLANWFPPPRMRTKREEIRNIILKLQEDQSKEKRKHKDTYSTEAPLGEGTEQYVNSISD",
# SENP7_HUMAN
"MDKRKLGRRPSSSEIITEGKRKKSSSDLSEIRKMLNAKPEDVHVQSPLSKFRSSERWTLPLQWERSLRNKVISLDHKNKKHIRGCPVTSKSSPERQLKVMLTNVLWTDLGRKFRKTLPRNDANLCDANKVQSDSLPSTSVDSLETCQKLEPLRQSLNLSERIPRVILTNVLGTELGRKYIRTPPVTEGSLSDTDNLQSEQLSSSSDGSLESYQNLNPHKSCYLSERGSQRSKTVDDNSAKQTAHNKEKRRKDDGISLLISDTQPEDLNSGSRGCDHLEQESRNKDVKYSDSKVELTLISRKTKRRLRNNLPDSQYCTSLDKSTEQTKKQEDDSTISTEFEKPSENYHQDPKLPEEITTKPTKSDFTKLSSLNSQELTLSNATKSASAGSTTETVENSNSIDIVGISSLVEKDENELNTIEKPILRGHNEGNQSLISAEPIVVSSDEEGPVEHKSSEILKLQSKQDRETTNENESTSESALLELPLITCESVQMSSELCPYNPVMENISSIMPSNEMDLQLDFIFTSVYIGKIKGASKGCVTITKKYIKIPFQVSLNEISLLVDTTHLKRFGLWKSKDDNHSKRSHAILFFWVSSDYLQEIQTQLEHSVLSQQSKSSEFIFLELHNPVSQREELKLKDIMTEISIISGELELSYPLSWVQAFPLFQNLSSKESSFIHYYCVSTCSFPAGVAVAEEMKLKSVSQPSNTDAAKPTYTFLQKQSSGCYSLSITSNPDEEWREVRHTGLVQKLIVYPPPPTKGGLGVTNEDLECLEEGEFLNDVIIDFYLKYLILEKASDELVERSHIFSSFFYKCLTRKENNLTEDNPNLSMAQRRHKRVRTWTRHINIFNKDYIFVPVNESSHWYLAVICFPWLEEAVYEDFPQTVSQQSQAQQSQNDNKTIDNDLRTTSTLSLSAEDSQSTESNMSVPKKMCKRPCILILDSLKAASVQNTVQNLREYLEVEWEVKLKTHRQFSKTNMVDLCPKVPKQDNSSDCGVYLLQYVESFFKDPIVNFELPIHLEKWFPRHVIKTKREDIRELILKLHLQQQKGSSS",
# SENP8_HUMAN
"MDPVVLSYMDSLLRQSDVSLLDPPSWLNDHIIGFAFEYFANSQFHDCSDHVSFISPEVTQFIKCTSNPAEIAMFLEPLDLPNKRVVFLAINDNSNQAAGGTHWSLLVYLQDKNSFFHYDSHSRSNSVHAKQVAEKLEAFLGRKGDKLAFVEEKAPAQQNSYDCGMYVICNTEALCQNFFRQQTESLLQLLTPAYITKKRGEWKDLITTLAKK",
# STABP_HUMAN
"MSDHGDVSLPPEDRVRALSQLGSAVEVNEDIPPRRYFRSGVEIIRMASIYSEEGNIEHAFILYNKYITLFIEKLPKHRDYKSAVIPEKKDTVKKLKEIAFPKAEELKAELLKRYTKEYTEYNEEKKKEAEELARNMAIQQELEKEKQRVAQQKQQQLEQEQFHAFEEMIRNQELEKERLKIVQEFGKVDPGLGGPLVPDLEKPSLDVFPTLTVSSIQPSDCHTTVRPAKPPVVDRSLKPGALSNSESIPTIDGLRHVVVPGRLCPQFLQLASANTARGVETCGILCGKLMRNEFTITHVLIPKQSAGSDYCNTENEEELFLIQDQQGLITLGWIHTHPTQTAFLSSVDLHTHCSYQMMLPESVAIVCSPKFQETGFFKLTDHGLEEISSCRQKGFHPHSKDPPLFCSCSHVTVVDRAVTITDLR",
# STALP_HUMAN
"MDQPFTVNSLKKLAAMPDHTDVSLSPEERVRALSKLGCNITISEDITPRRYFRSGVEMERMASVYLEEGNLENAFVLYNKFITLFVEKLPNHRDYQQCAVPEKQDIMKKLKEIAFPRTDELKNDLLKKYNVEYQEYLQSKNKYKAEILKKLEHQRLIEAERKRIAQMRQQQLESEQFLFFEDQLKKQELARGQMRSQQTSGLSEQIDGSALSCFSTHQNNSLLNVFADQPNKSDATNYASHSPPVNRALTPAATLSAVQNLVVEGLRCVVLPEDLCHKFLQLAESNTVRGIETCGILCGKLTHNEFTITHVIVPKQSAGPDYCDMENVEELFNVQDQHDLLTLGWIHTHPTQTAFLSSVDLHTHCSYQLMLPEAIAIVCSPKHKDTGIFRLTNAGMLEVSACKKKGFHPHTKEPRLFSICKHVLVKDIKIIVLDLR",
# UCHL1_HUMAN
"MQLKPMEINPEMLNKVLSRLGVAGQWRFVDVLGLEEESLGSVPAPACALLLLFPLTAQHENFRKKQIEELKGQEVSPKVYFMKQTIGNSCGTIGLIHAVANNQDKLGFEDGSVLKQFLSETEKMSPEDRAKCFEKNEAIQAAHDAVAQEGQCRVDDKVNFHFILFNNVDGHLYELDGRMPFPVNHGASSEDTLLKDAAKVCREFTEREQGEVRFSAVALCKAA",
# UCHL3_HUMAN
"MEGQRWLPLEANPEVTNQFLKQLGLHPNWQFVDVYGMDPELLSMVPRPVCAVLLLFPITEKYEVFRTEEEEKIKSQGQDVTSSVYFMKQTISNACGTIGLIHAIANNKDKMHFESGSTLKKFLEESVSMSPEERARYLENYDAIRVTHETSAHEGQTEAPSIDEKVDLHFIALVHVDGHLYELDGRKPFPINHGETSDETLLEDAIEVCKKFMERDPDELRFNAIALSAA",
# UCHL4_MOUSE
"MEGQRWLPLEANPEVTNQFLKQLGLHPNWQFVDVYGMESELLSIIPRPVCAVLLLFPITEKYEVFRTEEEEKIKSQGQDVTSSVYFMKQTISNACGTIGTIGLIHAIANNKDKVHFESGSTLKKFLEESVSMSPEERAKYLENYDAIRVTHETSAHEGQTEAPSIDEKVDLHFIALVHVDGHLYELDGWKPFPINHGKTSDETLLEDVIKVCKKFMERDPDELRFNAIALSAA",
# UCHL5_HUMAN
"MTGNAGEWCLMESDPGVFTELIKGFGCRGAQVEEIWSLEPENFEKLKPVHGLIFLFKWQPGEEPAGSVVQDSRLDTIFFAKQVINNACATQAIVSVLLNCTHQDVHLGETLSEFKEFSQSFDAAMKGLALSNSDVIRQVHNSFARQQMFEFDTKTSAKEEDAFHFVSYVPVNGRLYELDGLREGPIDLGACNQDDWISAVRPVIEKRIQKYSEGEIRFNLMAIVSDRKMIYEQKIAELQRQLAEEEPMDTDQGNSMLSAIQSEVAKNQMLIEEEVQKLKRYKIENIRRKHNYLPFIMELLKTLAEHQQLIPLVEKAKEKQNAKKAQETK",
# UBP1_HUMAN
"MPGVIPSESNGLSRGSPSKKNRLSLKFFQKKETKRALDFTDSQENEEKASEYRASEIDQVVPAAQSSPINCEKRENLLPFVGLNNLGNTCYLNSILQVLYFCPGFKSGVKHLFNIISRKKEALKDEANQKDKGNCKEDSLASYELICSLQSLIISVEQLQASFLLNPEKYTDELATQPRRLLNTLRELNPMYEGYLQHDAQEVLQCILGNIQETCQLLKKEEVKNVAELPTKVEEIPHPKEEMNGINSIEMDSMRHSEDFKEKLPKGNGKRKSDTEFGNMKKKVKLSKEHQSLEENQRQTRSKRKATSDTLESPPKIIPKYISENESPRPSQKKSRVKINWLKSATKQPSILSKFCSLGKITTNQGVKGQSKENECDPEEDLGKCESDNTTNGCGLESPGNTVTPVNVNEVKPINKGEEQIGFELVEKLFQGQLVLRTRCLECESLTERREDFQDISVPVQEDELSKVEESSEISPEPKTEMKTLRWAISQFASVERIVGEDKYFCENCHHYTEAERSLLFDKMPEVITIHLKCFAASGLEFDCYGGGLSKINTPLLTPLKLSLEEWSTKPTNDSYGLFAVVMHSGITISSGHYTASVKVTDLNSLELDKGNFVVDQMCEIGKPEPLNEEEARGVVENYNDEEVSIRVGGNTQPSKVLNKKNVEAIGLLGGQKSKADYELYNKASNPDKVASTAFAENRNSETSDTTGTHESDRNKESSDQTGINISGFENKISYVVQSLKEYEGKWLLFDDSEVKVTEEKDFLNSLSPSTSPTSTPYLLFYKKL",
# UBP10_HUMAN
"MALHSPQYIFGDFSPDEFNQFFVTPRSSVELPPYSGTVLCGTQAVDKLPDGQEYQRIEFGVDEVIEPSDTLPRTPSYSISSTLNPQAPEFILGCTASKITPDGITKEASYGSIDCQYPGSALALDGSSNVEAEVLENDGVSGGLGQRERKKKKKRPPGYYSYLKDGGDDSISTEALVNGHANSAVPNSVSAEDAEFMGDMPPSVTPRTCNSPQNSTDSVSDIVPDSPFPGALGSDTRTAGQPEGGPGADFGQSCFPAEAGRDTLSRTAGAQPCVGTDTTENLGVANGQILESSGEGTATNGVELHTTESIDLDPTKPESASPPADGTGSASGTLPVSQPKSWASLFHDSKPSSSSPVAYVETKYSPPAISPLVSEKQVEVKEGLVPVSEDPVAIKIAELLENVTLIHKPVSLQPRGLINKGNWCYINATLQALVACPPMYHLMKFIPLYSKVQRPCTSTPMIDSFVRLMNEFTNMPVPPKPRQALGDKIVRDIRPGAAFEPTYIYRLLTVNKSSLSEKGRQEDAEEYLGFILNGLHEEMLNLKKLLSPSNEKLTISNGPKNHSVNEEEQEEQGEGSEDEWEQVGPRNKTSVTRQADFVQTPITGIFGGHIRSVVYQQSSKESATLQPFFTLQLDIQSDKIRTVQDALESLVARESVQGYTTKTKQEVEISRRVTLEKLPPVLVLHLKRFVYEKTGGCQKLIKNIEYPVDLEISKELLSPGVKNKNFKCHRTYRLFAVVYHHGNSATGGHYTTDVFQIGLNGWLRIDDQTVKVINQYQVVKPTAERTAYLLYYRRVDLL",
# UBP11_HUMAN
"MAVAPRLFGGLCFRFRDQNPEVAVEGRLPISHSCVGCRRERTAMATVAANPAAAAAAVAAAAAVTEDREPQHEELPGLDSQWRQIENGESGRERPLRAGESWFLVEKHWYKQWEAYVQGGDQDSSTFPGCINNATLFQDEINWRLKEGLVEGEDYVLLPAAAWHYLVSWYGLEHGQPPIERKVIELPNIQKVEVYPVELLLVRHNDLGKSHTVQFSHTDSIGLVLRTARERFLVEPQEDTRLWAKNSEGSLDRLYDTHITVLDAALETGQLIIMETRKKDGTWPSAQLHVMNNNMSEEDEDFKGQPGICGLTNLGNTCFMNSALQCLSNVPQLTEYFLNNCYLEELNFRNPLGMKGEIAEAYADLVKQAWSGHHRSIVPHVFKNKVGHFASQFLGYQQHDSQELLSFLLDGLHEDLNRVKKKEYVELCDAAGRPDQEVAQEAWQNHKRRNDSVIVDTFHGLFKSTLVCPDCGNVSVTFDPFCYLSVPLPISHKRVLEVFFIPMDPRRKPEQHRLVVPKKGKISDLCVALSKHTGISPERMMVADVFSHRFYKLYQLEEPLSSILDRDDIFVYEVSGRIEAIEGSREDIVVPVYLRERTPARDYNNSYYGLMLFGHPLLVSVPRDRFTWEGLYNVLMYRLSRYVTKPNSDDEDDGDEKEDDEEDKDDVPGPSTGGSLRDPEPEQAGPSSGVTNRCPFLLDNCLGTSQWPPRRRRKQLFTLQTVNSNGTSDRTTSPEEVHAQPYIAIDWEPEMKKRYYDEVEAEGYVKHDCVGYVMKKAPVRLQECIELFTTVETLEKENPWYCPSCKQHQLATKKLDLWMLPEILIIHLKRFSYTKFSREKLDTLVEFPIRDLDFSEFVIQPQNESNPELYKYDLIAVSNHYGGMRDGHYTTFACNKDSGQWHYFDDNSVSPVNENQIESKAAYVLFYQRQDVARRLLSPAGSSGAPASPACSSPPSSEFMDVN",
# UBP12_HUMAN
"MEILMTVSKFASICTMGANASALEKEIGPEQFPVNEHYFGLVNFGNTCYCNSVLQALYFCRPFREKVLAYKSQPRKKESLLTCLADLFHSIATQKKKVGVIPPKKFITRLRKENELFDNYMQQDAHEFLNYLLNTIADILQEERKQEKQNGRLPNGNIDNENNNSTPDPTWVHEIFQGTLTNETRCLTCETISSKDEDFLDLSVDVEQNTSITHCLRGFSNTETLCSEYKYYCEECRSKQEAHKRMKVKKLPMILALHLKRFKYMDQLHRYTKLSYRVVFPLELRLFNTSGDATNPDRMYDLVAVVVHCGSGPNRGHYIAIVKSHDFWLLFDDDIVEKIDAQAIEEFYGLTSDISKNSESGYILFYQSRD",
# UBP13_HUMAN
"MQRRGALFGMPGGSGGRKMAAGDIGELLVPHMPTIRVPRSGDRVYKNECAFSYDSPNSEGGLYVCMNTFLAFGREHVERHFRKTGQSVYMHLKRHVREKVRGASGGALPKRRNSKIFLDLDTDDDLNSDDYEYEDEAKLVIFPDHYEIALPNIEELPALVTIACDAVLSSKSPYRKQDPDTWENELPVSKYANNLTQLDNGVRIPPSGWKCARCDLRENLWLNLTDGSVLCGKWFFDSSGGNGHALEHYRDMGYPLAVKLGTITPDGADVYSFQEEEPVLDPHLAKHLAHFGIDMLHMHGTENGLQDNDIKLRVSEWEVIQESGTKLKPMYGPGYTGLKNLGNSCYLSSVMQAIFSIPEFQRAYVGNLPRIFDYSPLDPTQDFNTQMTKLGHGLLSGQYSKPPVKSELIEQVMKEEHKPQQNGISPRMFKAFVSKSHPEFSSNRQQDAQEFFLHLVNLVERNRIGSENPSDVFRFLVEERIQCCQTRKVRYTERVDYLMQLPVAMEAATNKDELIAYELTRREAEANRRPLPELVRAKIPFSACLQAFSEPENVDDFWSSALQAKSAGVKTSRFASFPEYLVVQIKKFTFGLDWVPKKFDVSIDMPDLLDINHLRARGLQPGEEELPDISPPIVIPDDSKDRLMNQLIDPSDIDESSVMQLAEMGFPLEACRKAVYFTGNMGAEVAFNWIIVHMEEPDFAEPLTMPGYGGAASAGASVFGASGLDNQPPEEIVAIITSMGFQRNQAIQALRATNNNLERALDWIFSHPEFEEDSDFVIEMENNANANIISEAKPEGPRVKDGSGTYELFAFISHMGTSTMSGHYICHIKKEGRWVIYNDHKVCASERPPKDLGYMYFYRRIPS",
# UBP14_HUMAN
"MPLYSVTVKWGKEKFEGVELNTDEPPMVFKAQLFALTGVQPARQKVMVKGGTLKDDDWGNIKIKNGMTLLMMGSADALPEEPSAKTVFVEDMTEEQLASAMELPCGLTNLGNTCYMNATVQCIRSVPELKDALKRYAGALRASGEMASAQYITAALRDLFDSMDKTSSSIPPIILLQFLHMAFPQFAEKGEQGQYLQQDANECWIQMMRVLQQKLEAIEDDSVKETDSSSASAATPSKKKSLIDQFFGVEFETTMKCTESEEEEVTKGKENQLQLSCFINQEVKYLFTGLKLRLQEEITKQSPTLQRNALYIKSSKISRLPAYLTIQMVRFFYKEKESVNAKVLKDVKFPLMLDMYELCTPELQEKMVSFRSKFKDLEDKKVNQQPNTSDKKSSPQKEVKYEPFSFADDIGSNNCGYYDLQAVLTHQGRSSSSGHYVSWVKRKQDEWIKFDDDKVSIVTPEDILRLSGGGDWHIAYVLLYGPRRVEIMEEESEQ",
# UBP15_HUMAN
"MAEGGAADLDTQRSDIATLLKTSLRKGDTWYLVDSRWFKQWKKYVGFDSWDKYQMGDQNVYPGPIDNSGLLKDGDAQSLKEHLIDELDYILLPTEGWNKLVSWYTLMEGQEPIARKVVEQGMFVKHCKVEVYLTELKLCENGNMNNVVTRRFSKADTIDTIEKEIRKIFSIPDEKETRLWNKYMSNTFEPLNKPDSTIQDAGLYQGQVLVIEQKNEDGTWPRGPSTPKSPGASNFSTLPKISPSSLSNNYNNMNNRNVKNSNYCLPSYTAYKNYDYSEPGRNNEQPGLCGLSNLGNTCFMNSAIQCLSNTPPLTEYFLNDKYQEELNFDNPLGMRGEIAKSYAELIKQMWSGKFSYVTPRAFKTQVGRFAPQFSGYQQQDCQELLAFLLDGLHEDLNRIRKKPYIQLKDADGRPDKVVAEEAWENHLKRNDSIIVDIFHGLFKSTLVCPECAKISVTFDPFCYLTLPLPMKKERTLEVYLVRMDPLTKPMQYKVVVPKIGNILDLCTALSALSGIPADKMIVTDIYNHRFHRIFAMDENLSSIMERDDIYVFEININRTEDTEHVIIPVCLREKFRHSSYTHHTGSSLFGQPFLMAVPRNNTEDKLYNLLLLRMCRYVKISTETEETEGSLHCCKDQNINGNGPNGIHEEGSPSEMETDEPDDESSQDQELPSENENSQSEDSVGGDNDSENGLCTEDTCKGQLTGHKKRLFTFQFNNLGNTDINYIKDDTRHIRFDDRQLRLDERSFLALDWDPDLKKRYFDENAAEDFEKHESVEYKPPKKPFVKLKDCIELFTTKEKLGAEDPWYCPNCKEHQQATKKLDLWSLPPVLVVHLKRFSYSRYMRDKLDTLVDFPINDLDMSEFLINPNAGPCRYNLIAVSNHYGGMGGGHYTAFAKNKDDGKWYYFDDSSVSTASEDQIVSKAAYVLFYQRQDTFSGTGFFPLDRETKGASAATGIPLESDEDSNDNDNDIENENCMHTN",
# UBP16_HUMAN
"MGKKRTKGKTVPIDDSSETLEPVCRHIRKGLEQGNLKKALVNVEWNICQDCKTDNKVKDKAEEETEEKPSVWLCLKCGHQGCGRNSQEQHALKHYLTPRSEPHCLVLSLDNWSVWCYVCDNEVQYCSSNQLGQVVDYVRKQASITTPKPAEKDNGNIELENKKLEKESKNEQEREKKENMAKENPPMNSPCQITVKGLSNLGNTCFFNAVMQNLSQTPVLRELLKEVKMSGTIVKIEPPDLALTEPLEINLEPPGPLTLAMSQFLNEMQETKKGVVTPKELFSQVCKKAVRFKGYQQQDSQELLRYLLDGMRAEEHQRVSKGILKAFGNSTEKLDEELKNKVKDYEKKKSMPSFVDRIFGGELTSMIMCDQCRTVSLVHESFLDLSLPVLDDQSGKKSVNDKNLKKTVEDEDQDSEEEKDNDSYIKERSDIPSGTSKHLQKKAKKQAKKQAKNQRRQQKIQGKVLHLNDICTIDHPEDSEYEAEMSLQGEVNIKSNHISQEGVMHKEYCVNQKDLNGQAKMIESVTDNQKSTEEVDMKNINMDNDLEVLTSSPTRNLNGAYLTEGSNGEVDISNGFKNLNLNAALHPDEINIEILNDSHTPGTKVYEVVNEDPETAFCTLANREVFNTDECSIQHCLYQFTRNEKLRDANKLLCEVCTRRQCNGPKANIKGERKHVYTNAKKQMLISLAPPVLTLHLKRFQQAGFNLRKVNKHIKFPEILDLAPFCTLKCKNVAEENTRVLYSLYGVVEHSGTMRSGHYTAYAKARTANSHLSNLVLHGDIPQDFEMESKGQWFHISDTHVQAVPTTKVLNSQAYLLFYERIL",
# U17LA_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKPPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYKPPLANYMLFREHSQTCHRHKGCMLCTMQAHITRALHIPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMRKACLPGHKQVDRHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHNSAKVLILVLKRFPDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYSSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGVEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRRVEGTVPPDVLVIHQSKYKCRMKNHHPEQQSSLLNLSSTTPTDQESMNTGTLASLRGRTRRSKGKNKHSKRALLVCQ",
# U17LB_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQTNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# UL17C_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSNRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKMLTLLTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLKLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LD_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRLDAAFAEIQRTSLPEKSPLSCETRVDLCDDLVPEARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHPSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTAASITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDRWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LF_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIDKNVQYPECLDMKLYMSQTNSGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQWSQWKYRPTRRGAHTHAHTQTHT",
# U17LH_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTAASITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LI_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQTNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRAKQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LJ_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQTNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLKLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LK_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LL_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSNRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKMLTLLTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LM_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLKLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17L3_HUMAN
"MGDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSSETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYENASLQCLTYTLPLANYMLSREHSQTCQRPKCCMLCTMQAHITWALHSPGHVIQPSQALASGFHRGKQEDVHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGCWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVKQALEQLVKPEELNGENAYHCGLCLQRAPASNTLTLHTSAKVLILVLKRFSDVAGNKLAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHDGHYFSYVKAQEGQWYKMDDAEVTVCSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRAKQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVGKVEGTLPPNALVIHQSKYKCGMKNHHPEQQSSLLNLSSTTRTDQESMNTGTLASLQGRTRRAKGKNKHSKRALLVCQ",
# U17L4_HUMAN
"MGDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSSETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYENASLQCLTYTLPLANYMLSREHSQTCQRPKCCMLCTMQAHITWALHSPGHVIQPSQALAAGFHRGKQEDVHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGCWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVKQALEQLVKPEELNGENAYHCGLCLQRAPASNTLTLHTSAKVLILVLKRFSDVAGNKLAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHDGYYFSYVKAQEGQWYKMDDAEVTVCSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRPATQGELKRDHPCLQVPELDEHLVERATEESTLDHWKFPQEQNKMKPEFNVRKVEGTLPPNVLVIHQSKYKCGMKNHHPEQQSSLLNLSSMNSTDQESMNTGTLASLQGRTRRSKGKNKHSKRSLLVCQ",
# U17L5_HUMAN
"MEDDSLYLRGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLAKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17L7_HUMAN
"MEDDSLYLGGDWQFNHFSKLTSSRLDAAFAEIQRTSLSEKSPLSSETRFDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQKIGNTFYVNVSLQCLTYTLPLSNYMLSREDSQTCHLHKCCMFCTMQAHITWALHSPGHVIQPSQVLAAGFHRGEQEDAHEFLMFTVDAMKKACLPGHKQLDHHSKDTTLIHQIFGAYWRSQIKYLHCHGVSDTFDPYLDIALDIQAAQSVKQALEQLVKPKELNGENAYHCGLCLQKAPASKTLTLPTSAKVLILVLKRFSDVTGNKLAKNVQYPKCRDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASGITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRPATQGELKRDHPCLQVPELDEHLVERATQESTLDHWKFPQEQNKTKPEFNVRKVEGTLPPNVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTKPTDQESMNTGTLASLQGSTRRSKGNNKHSKRSLLVCQ",
# U17L8_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSPRPDAAFAEIQRTSLPEKSPLSSETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYLNASLQCLTYTPPLANYMLSREHSQTCQRPKCCMLCTMQAHITWALHSPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGCWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVKQALEQLVKPEELNGENAYPCGLCLQRAPASNTLTLHTSAKVLILVLKRFCDVTGNKLAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGYYFSYVKAQEGQWYKMDDAEVTACSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRPATQGELKRDHPCLQVPELDEHLVERATEESTLDHWKFPQEQNKMKPEFNVRKVEGTLPPNVLVIHQSKYKCGMKNHHPEQQSSLLNLSSMNSTDQESMNTGTLASLQGRTRRSKGKNKHSKRSLLVCQ",
# U17PA_MOUSE
"MVVALSFPEADPALSSPDAPELHQDEAQVVEELTVNGKHSLSWESPQGPGCGLQNTGNSCYLNAALQCLTHTPPLADYMLSQEHSQTCCSPEGCKLCAMEALVTQSLLHSHSGDVMKPSHILTSAFHKHQQEDAHEFLMFTLETMHESCLQVHRQSKPTSEDSSPIHDIFGGWWRSQIKCLLCQGTSDTYDRFLDIPLDISSAQSVKQALWDTEKSEELCGDNAYYCGKCRQKMPASKTLHVHIAPKVLMVVLNRFSAFTGNKLDRKVSYPEFLDLKPYLSEPTGGPLPYALYAVLVHDGATSHSGHYFCCVKAGHGKWYKMDDTKVTRCDVTSVLNENAYVLFYVQQANLKQVSIDMPEGRINEVLDPEYQLKKSRRKKHKKKSPFTEDLGEPCENRDKRAIKETSLGKGKVLQEVNHKKAGQKHGNTKLMPQKQNHQKAGQNLRNTEVELDLPADAIVIHQPRSTANWGRDSPDKENQPLHNADRLLTSQGPVNTWQLCRQEGRRRSKKGQNKNKQGQRLLLVC",
# U17PB_MOUSE
"MVVALSFPEADPAMSPPSAPELHQDEAQVVEELAANGKHSLSWESPQGPGCGLQNTGNSCYLNAALQCLTHTPPLADYMLSQEHSQTCCSPEGCKMCAMEAHVTQSLLHTHSGDVMKPSQNLTSAFHKRKQEDAHEFLMFTLETMHESCLQVHRQSEPTSEDSSPIHDIFGGWWRSQIKCHHCQGTSYSYDPFLDIPLDISSVQSVKQALQDTEKAEELCGENSYYCGRCRQKKPASKTLKLYSAPKVLMLVLKRFSGSMGKKLDRKVSYPEFLDLKPYLSQPTGGPLPYALYAVLVHEGATCHSGHYFCCVKAGHGKWYKMDDTKVTSCDVTSVLNENAYVLFYVQQNDLKKGSINMPEGRIHEVLDAKYQLKKSGEKKHNKSPCTEDAGEPCENREKRSSKETSLGEGKVLQEQDHQKAGQKQENTKLTPQEQNHEKGGQNLRNTEGELDRLSGAIVVYQPICTAN",
# U17LC_MOUSE
"MVVSLSFPEADPALSSPGAQQLHQDEAQVVVELTANDKPSLSWECPQGPGCGLQNTGNSCYLNAALQCLTHTPPLADYMLSQEYSQTCCSPEGCKMCAMEAHVTQSLLHSHSGDVMKPSQILTSAFHKHQQEDAHEFLMFTLETMHESCLQVHRQSEPTSEDSSPIHDIFGGLWRSQIKCLHCQGTSDTYDRFLDVPLDISSAQSVNQALWDTEKSEELRGENAYYCGRCRQKMPASKTLHIHSAPKVLLLVLKRFSAFMGNKLDRKVSYPEFLDLKPYLSQPTGGPLPYALYAVLVHEGATCHSGHYFSYVKARHGAWYKMDDTKVTSCDVTSVLNENAYVLFYVQQTDLKQVSIDMPEGRVHEVLDPEYQLKKSRRKKHKKKSPCTEDAGEPCKNREKRATKETSLGEGKVLQEKNHKKAGQKHENTKLVPQEQNHQKLGQKHRINEILPQEQNHQKAGQSLRNTEGELDLPADAIVIHLLRSTENWGRDAPDKENQPWHNADRLLTSQDPVNTGQLCRQEGRRRSKKGKNKNKQGQRLLLVC",
# U17PD_MOUSE
"MVVSLSFPEADPALSSPGAQQLHQDEAQVVVELTANDKPSLSWECPQGPGCGLQNTGNSCYLNAALQCLTHTPPLADYMLSQEYSQTCCSPEGCKMCAMEAHVTQSLLHSHSGDVMKPSQILTSAFHKHQQEDAHEFLMFTLETMHESCLQVHRQSEPTSEDSSPIHDIFGGLWRSQIKCLHCQGTSDTYDRFLDVPLDISSAQSVNQALWDTEKSEELRGENAYYCGRCRQKMPASKTLHIHSAPKVLLLVLKRFSAFMGNKLDRKVSYPEFLDLKPYLSQPTGGPLPYALYAVLVHEGATCHSGHYFSYVKAGHGKWYKMDDTKVTSCDVTSVLNENAYVLFYVQQTDLKEVSIDMPEGRIHEVLDPEYQLKKSRRKKHKKKSPCTEDVGEPSKNREKKATKETSLGEGKVLQEKNHKKAGQKHENTKLVPQEQNHQKLGQKHRNNEILPQEQNHQKTGQSLRNTEGELDLPADAIVIHLPRSIANWGRDTPDKVNQPWHNADRLLTSQDLVNTGQLCRQEGRRRSKKGKNKNKQGQKLLLVR",
# U17PE_MOUSE
"MVVSLSFPEETGGENLPSAPLEDSSKFFEEVFGDMVFARSFPEADPALSSPDAPELHQDEAQVVEELTTNGKHSLSWESPQGPGCGLQNTGNSCYLNAALQCLTHTPPLADYMLSQEHSQTCCSPEGCKMCAMEAHVTQSLLHSHSGDVMKPSQILTSAFHKHQQEDAHEFLMFTLETMHESCLQVHRQSDPTPQDTSPIHDIFGGWWRSQIKCLHCQGTSHTFDPFLDVPLDISSAQSVNQALWDTGKSEELLGENAYYCGRCRQKMPASKTLHVHIAPKVLLLVLKRFSAFTGNKLDRKVSYPEFLDLKPYLSEPTGGPLPYALYAVLVHDGATSNSGHYFCCVKAGHGKWYKMDDTKVTRCDVTSVLNENAYVLFYVQQTDLKQVSIDMPEGRVHEVLDPKYQLKKSRRKKRKKQCHCTDDAGEACENREKRAKKETSLGEGKVPQEVNHEKAGQKHGNTKLVPQEQNHQRAGQNLRNTEVELDLPVDAIVIHQPRSTANWGTDAPDKENQPWHNGDRLLTSQGLMSPGQLCSQGGR",
# UBP18_HUMAN
"MSKAFGLLRQICQSILAESSQSPADLEEKKEEDSNMKREQPRERPRAWDYPHGLVGLHNIGQTCCLNSLIQVFVMNVDFTRILKRITVPRGADEQRRSVPFQMLLLLEKMQDSRQKAVRPLELAYCLQKCNVPLFVQHDAAQLYLKLWNLIKDQITDVHLVERLQALYTIRVKDSLICVDCAMESSRNSSMLTLPLSLFDVDSKPLKTLEDALHCFFQPRELSSKSKCFCENCGKKTRGKQVLKLTHLPQTLTIHLMRFSIRNSQTRKICHSLYFPQSLDFSQILPMKRESCDAEEQSGGQYELFAVIAHVGMADSGHYCVYIRNAVDGKWFCFNDSNICLVSWEDIQCTYGNPNYHWQETAYLLVYMKMEC",
# UBP19_HUMAN
"MSGGASATGPRRGPPGLEDTTSKKKQKDRANQESKDGDPRKETGSRYVAQAGLEPLASGDPSASASHAAGITGSRHRTRLFFPSSSGSASTPQEEQTKEGACEDPHDLLATPTPELLLDWRQSAEEVIVKLRVGVGPLQLEDVDAAFTDTDCVVRFAGGQQWGGVFYAEIKSSCAKVQTRKGSLLHLTLPKKVPMLTWPSLLVEADEQLCIPPLNSQTCLLGSEENLAPLAGEKAVPPGNDPVSPAMVRSRNPGKDDCAKEEMAVAADAATLVDEPESMVNLAFVKNDSYEKGPDSVVVHVYVKEICRDTSRVLFREQDFTLIFQTRDGNFLRLHPGCGPHTTFRWQVKLRNLIEPEQCTFCFTASRIDICLRKRQSQRWGGLEAPAARVGGAKVAVPTGPTPLDSTPPGGAPHPLTGQEEARAVEKDKSKARSEDTGLDSVATRTPMEHVTPKPETHLASPKPTCMVPPMPHSPVSGDSVEEEEEEEKKVCLPGFTGLVNLGNTCFMNSVIQSLSNTRELRDFFHDRSFEAEINYNNPLGTGGRLAIGFAVLLRALWKGTHHAFQPSKLKAIVASKASQFTGYAQHDAQEFMAFLLDGLHEDLNRIQNKPYTETVDSDGRPDEVVAEEAWQRHKMRNDSFIVDLFQGQYKSKLVCPVCAKVSITFDPFLYLPVPLPQKQKVLPVFYFAREPHSKPIKFLVSVSKENSTASEVLDSLSQSVHVKPENLRLAEVIKNRFHRVFLPSHSLDTVSPSDTLLCFELLSSELAKERVVVLEVQQRPQVPSVPISKCAACQRKQQSEDEKLKRCTRCYRVGYCNQLCQKTHWPDHKGLCRPENIGYPFLVSVPASRLTYARLAQLLEGYARYSVSVFQPPFQPGRMALESQSPGCTTLLSTGSLEAGDSERDPIQPPELQLVTPMAEGDTGLPRVWAAPDRGPVPSTSGISSEMLASGPIEVGSLPAGERVSRPEAAVPGYQHPSEAMNAHTPQFFIYKIDSSNREQRLEDKGDTPLELGDDCSLALVWRNNERLQEFVLVASKELECAEDPGSAGEAARAGHFTLDQCLNLFTRPEVLAPEEAWYCPQCKQHREASKQLLLWRLPNVLIVQLKRFSFRSFIWRDKINDLVEFPVRNLDLSKFCIGQKEEQLPSYDLYAVINHYGGMIGGHYTACARLPNDRSSQRSDVGWRLFDDSTVTTVDESQVVTRYAYVLFYRRRNSPVERPPRAGHSEHHPDLGPAAEAAASQASRIWQELEAEEEPVPEGSGPLGPWGPQDWVGPLPRGPTTPDEGCLRYFVLGTVAALVALVLNVFYPLVSQSRWR",
# UBP2_HUMAN
"MSQLSSTLKRYTESARYTDAHYAKSGYGAYTPSSYGANLAASLLEKEKLGFKPVPTSSFLTRPRTYGPSSLLDYDRGRPLLRPDITGGGKRAESQTRGTERPLGSGLSGGSGFPYGVTNNCLSYLPINAYDQGVTLTQKLDSQSDLARDFSSLRTSDSYRIDPRNLGRSPMLARTRKELCTLQGLYQTASCPEYLVDYLENYGRKGSASQVPSQAPPSRVPEIISPTYRPIGRYTLWETGKGQAPGPSRSSSPGRDGMNSKSAQGLAGLRNLGNTCFMNSILQCLSNTRELRDYCLQRLYMRDLHHGSNAHTALVEEFAKLIQTIWTSSPNDVVSPSEFKTQIQRYAPRFVGYNQQDAQEFLRFLLDGLHNEVNRVTLRPKSNPENLDHLPDDEKGRQMWRKYLEREDSRIGDLFVGQLKSSLTCTDCGYCSTVFDPFWDLSLPIAKRGYPEVTLMDCMRLFTKEDVLDGDEKPTCCRCRGRKRCIKKFSIQRFPKILVLHLKRFSESRIRTSKLTTFVNFPLRDLDLREFASENTNHAVYNLYAVSNHSGTTMGGHYTAYCRSPGTGEWHTFNDSSVTPMSSSQVRTSDAYLLFYELASPPSRM",
# UBP20_HUMAN
"MGDSRDLCPHLDSIGEVTKEDLLLKSKGTCQSCGVTGPNLWACLQVACPYVGCGESFADHSTIHAQAKKHNLTVNLTTFRLWCYACEKEVFLEQRLAAPLLGSSSKFSEQDSPPPSHPLKAVPIAVADEGESESEDDDLKPRGLTGMKNLGNSCYMNAALQALSNCPPLTQFFLECGGLVRTDKKPALCKSYQKLVSEVWHKKRPSYVVPTSLSHGIKLVNPMFRGYAQQDTQEFLRCLMDQLHEELKEPVVATVALTEARDSDSSDTDEKREGDRSPSEDEFLSCDSSSDRGEGDGQGRGGGSSQAETELLIPDEAGRAISEKERMKDRKFSWGQQRTNSEQVDEDADVDTAMAALDDQPAEAQPPSPRSSSPCRTPEPDNDAHLRSSSRPCSPVHHHEGHAKLSSSPPRASPVRMAPSYVLKKAQVLSAGSRRRKEQRYRSVISDIFDGSILSLVQCLTCDRVSTTVETFQDLSLPIPGKEDLAKLHSAIYQNVPAKPGACGDSYAAQGWLAFIVEYIRRFVVSCTPSWFWGPVVTLEDCLAAFFAADELKGDNMYSCERCKKLRNGVKYCKVLRLPEILCIHLKRFRHEVMYSFKINSHVSFPLEGLDLRPFLAKECTSQITTYDLLSVICHHGTAGSGHYIAYCQNVINGQWYEFDDQYVTEVHETVVQNAEGYVLFYRKSSEEAMRERQQVVSLAAMREPSLLRFYVSREWLNKFNTFAEPGPITNQTFLCSHGGIPPHKYHYIDDLVVILPQNVWEHLYNRFGGGPAVNHLYVCSICQVEIEALAKRRRIEIDTFIKLNKAFQAEESPGVIYCISMQWFREWEAFVKGKDNEPPGPIDNSRIAQVKGSGHVQLKQGADYGQISEETWTYLNSLYGGGPEIAIRQSVAQPLGPENLHGEQKIEAETRAV",
# UBP21_HUMAN
"MPQASEHRLGRTREPPVNIQPRVGSKLPFAPRARSKERRNPASGPNPMLRPLPPRPGLPDERLKKLELGRGRTSGPRPRGPLRADHGVPLPGSPPPTVALPLPSRTNLARSKSVSSGDLRPMGIALGGHRGTGELGAALSRLALRPEPPTLRRSTSLRRLGGFPGPPTLFSIRTEPPASHGSFHMISARSSEPFYSDDKMAHHTLLLGSGHVGLRNLGNTCFLNAVLQCLSSTRPLRDFCLRRDFRQEVPGGGRAQELTEAFADVIGALWHPDSCEAVNPTRFRAVFQKYVPSFSGYSQQDAQEFLKLLMERLHLEINRRGRRAPPILANGPVPSPPRRGGALLEEPELSDDDRANLMWKRYLEREDSKIVDLFVGQLKSCLKCQACGYRSTTFEVFCDLSLPIPKKGFAGGKVSLRDCFNLFTKEEELESENAPVCDRCRQKTRSTKKLTVQRFPRILVLHLNRFSASRGSIKKSSVGVDFPLQRLSLGDFASDKAGSPVYQLYALCNHSGSVHYGHYTALCRCQTGWHVYNDSRVSPVSENQVASSEGYVLFYQLMQEPPRCL",
# UBP22_HUMAN
"MVSRPEPEGEAMDAELAVAPPGCSHLGSFKVDNWKQNLRAIYQCFVWSGTAEARKRKAKSCICHVCGVHLNRLHSCLYCVFFGCFTKKHIHEHAKAKRHNLAIDLMYGGIYCFLCQDYIYDKDMEIIAKEEQRKAWKMQGVGEKFSTWEPTKRELELLKHNPKRRKITSNCTIGLRGLINLGNTCFMNCIVQALTHTPLLRDFFLSDRHRCEMQSPSSCLVCEMSSLFQEFYSGHRSPHIPYKLLHLVWTHARHLAGYEQQDAHEFLIAALDVLHRHCKGDDNGKKANNPNHCNCIIDQIFTGGLQSDVTCQVCHGVSTTIDPFWDISLDLPGSSTPFWPLSPGSEGNVVNGESHVSGTTTLTDCLRRFTRPEHLGSSAKIKCSGCHSYQESTKQLTMKKLPIVACFHLKRFEHSAKLRRKITTYVSFPLELDMTPFMASSKESRMNGQYQQPTDSLNNDNKYSLFAVVNHQGTLESGHYTSFIRQHKDQWFKCDDAIITKASIKDVLDSEGYLLFYHKQFLEYE",
# UBP24_HUMAN
"MESEEEQHMTTLLCMGFSDPATIRKALRLAKNDINEAVALLTNERPGLDYGGYEPMDSGGGPSPGPGGGPRGDGGGDGGGGGPSRGGSTGGGGGFDPPPAYHEVVDAEKNDENGNCSGEGIEFPTTNLYELESRVLTDHWSIPYKREESLGKCLLASTYLARLGLSESDENCRRFMDRCMPEAFKKLLTSSAVHKWGTEIHEGIYNMLMLLIELVAERIKQDPIPTGLLGVLTMAFNPDNEYHFKNRMKVSQRNWAEVFGEGNMFAVSPVSTFQKEPHGWVVDLVNKFGELGGFAAIQAKLHSEDIELGAVSALIQPLGVCAEYLNSSVVQPMLDPVILTTIQDVRSVEEKDLKDKRLVSIPELLSAVKLLCMRFQPDLVTIVDDLRLDILLRMLKSPHFSAKMNSLKEVTKLIEDSTLSKSVKNAIDTDRLLDWLVENSVLSIALEGNIDQAQYCDRIKGIIELLGSKLSLDELTKIWKIQSGQSSTVIENIHTIIAAAAVKFNSDQLNHLFVLIQKSWETESDRVRQKLLSLIGRIGREARFETTSGKVLDVLWELAHLPTLPSSLIQQALEEHLTILSDAYAVKEAIKRSYIIKCIEDIKRPGEWSGLEKNKKDGFKSSQLNNPQFVWVVPALRQLHEITRSFIKQTYQKQDKSIIQDLKKNFEIVKLVTGSLIACHRLAAAVAGPGGLSGSTLVDGRYTYREYLEAHLKFLAFFLQEATLYLGWNRAKEIWECLVTGQDVCELDREMCFEWFTKGQHDLESDVQQQLFKEKILKLESYEITMNGFNLFKTFFENVNLCDHRLKRQGAQLYVEKLELIGMDFIWKIAMESPDEEIANEAIQLIINYSYINLNPRLKKDSVSLHKKFIADCYTRLEAASSALGGPTLTHAVTRATKMLTATAMPTVATSVQSPYRSTKLVIIERLLLLAERYVITIEDFYSVPRTILPHGASFHGHLLTLNVTYESTKDTFTVEAHSNETIGSVRWKIAKQLCSPVDNIQIFTNDSLLTVNKDQKLLHQLGFSDEQILTVKTSGSGTPSGSSADSSTSSSSSSSGVFSSSYAMEQEKSLPGVVMALVCNVFDMLYQLANLEEPRITLRVRKLLLLIPTDPAIQEALDQLDSLGRKKTLLSESSSQSSKSPSLSSKQQHQPSASSILESLFRSFAPGMSTFRVLYNLEVLSSKLMPTADDDMARSCAKSFCENFLKAGGLSLVVNVMQRDSIPSEVDYETRQGVYSICLQLARFLLVGQTMPTLLDEDLTKDGIEALSSRPFRNVSRQTSRQMSLCGTPEKSSYRQLSVSDRSSIRVEEIIPAARVAIQTMEVSDFTSTVACFMRLSWAAAAGRLDLVGSSQPIKESNSLCPAGIRNRLSSSGSNCSSGSEGEPVALHAGICVRQQSVSTKDSLIAGEALSLLVTCLQLRSQQLASFYNLPCVADFIIDILLGSPSAEIRRVACDQLYTLSQTDTSAHPDVQKPNQFLLGVILTAQLPLWSPTSIMRGVNQRLLSQCMEYFDLRCQLLDDLTTSEMEQLRISPATMLEDEITWLDNFEPNRTAECETSEADNILLAGHLRLIKTLLSLCGAEKEMLGSSLIKPLLDDFLFRASRIILNSHSPAGSAAISQQDFHPKCSTANSRLAAYEVLVMLADSSPSNLQIIIKELLSMHHQPDPALTKEFDYLPPVDSRSSSGFVGLRNGGATCYMNAVFQQLYMQPGLPESLLSVDDDTDNPDDSVFYQVQSLFGHLMESKLQYYVPENFWKIFKMWNKELYVREQQDAYEFFTSLIDQMDEYLKKMGRDQIFKNTFQGIYSDQKICKDCPHRYEREEAFMALNLGVTSCQSLEISLDQFVRGEVLEGSNAYYCEKCKEKRITVKRTCIKSLPSVLVIHLMRFGFDWESGRSIKYDEQIRFPWMLNMEPYTVSGMARQDSSSEVGENGRSVDQGGGGSPRKKVALTENYELVGVIVHSGQAHAGHYYSFIKDRRGCGKGKWYKFNDTVIEEFDLNDETLEYECFGGEYRPKVYDQTNPYTDVRRRYWNAYMLFYQRVSDQNSPVLPKKSRVSVVRQEAEDLSLSAPSSPEISPQSSPRPHRPNNDRLSILTKLVKKGEKKGLFVEKMPARIYQMVRDENLKFMKNRDVYSSDYFSFVLSLASLNATKLKHPYYPCMAKVSLQLAIQFLFQTYLRTKKKLRVDTEEWIATIEALLSKSFDACQWLVEYFISSEGRELIKIFLLECNVREVRVAVATILEKTLDSALFYQDKLKSLHQLLEVLLALLDKDVPENCKNCAQYFFLFNTFVQKQGIRAGDLLLRHSALRHMISFLLGASRQNNQIRRWSSAQAREFGNLHNTVALLVLHSDVSSQRNVAPGIFKQRPPISIAPSSPLLPLHEEVEALLFMSEGKPYLLEVMFALRELTGSLLALIEMVVYCCFCNEHFSFTMLHFIKNQLETAPPHELKNTFQLLHEILVIEDPIQVERVKFVFETENGLLALMHHSNHVDSSRCYQCVKFLVTLAQKCPAAKEYFKENSHHWSWAVQWLQKKMSEHYWTPQSNVSNETSTGKTFQRTISAQDTLAYATALLNEKEQSGSSNGSESSPANENGDRHLQQGSESPMMIGELRSDLDDVDP",
# UBP25_HUMAN
"MTVEQNVLQQSAAQKHQQTFLNQLREITGINDTQILQQALKDSNGNLELAVAFLTAKNAKTPQQEETTYYQTALPGNDRYISVGSQADTNVIDLTGDDKDDLQRAIALSLAESNRAFRETGITDEEQAISRVLEASIAENKACLKRTPTEVWRDSRNPYDRKRQDKAPVGLKNVGNTCWFSAVIQSLFNLLEFRRLVLNYKPPSNAQDLPRNQKEHRNLPFMRELRYLFALLVGTKRKYVDPSRAVEILKDAFKSNDSQQQDVSEFTHKLLDWLEDAFQMKAEEETDEEKPKNPMVELFYGRFLAVGVLEGKKFENTEMFGQYPLQVNGFKDLHECLEAAMIEGEIESLHSENSGKSGQEHWFTELPPVLTFELSRFEFNQALGRPEKIHNKLEFPQVLYLDRYMHRNREITRIKREEIKRLKDYLTVLQQRLERYLSYGSGPKRFPLVDVLQYALEFASSKPVCTSPVDDIDASSPPSGSIPSQTLPSTTEQQGALSSELPSTSPSSVAAISSRSVIHKPFTQSRIPPDLPMHPAPRHITEEELSVLESCLHRWRTEIENDTRDLQESISRIHRTIELMYSDKSMIQVPYRLHAVLVHEGQANAGHYWAYIFDHRESRWMKYNDIAVTKSSWEELVRDSFGGYRNASAYCLMYINDKAQFLIQEEFNKETGQPLVGIETLPPDLRDFVEEDNQRFEKELEEWDAQLAQKALQEKLLASQKLRESETSVTTAQAAGDPEYLEQPSRSDFSKHLKEETIQIITKASHEHEDKSPETVLQSAIKLEYARLVKLAQEDTPPETDYRLHHVVVYFIQNQAPKKIIEKTLLEQFGDRNLSFDERCHNIMKVAQAKLEMIKPEEVNLEEYEEWHQDYRKFRETTMYLIIGLENFQRESYIDSLLFLICAYQNNKELLSKGLYRGHDEELISHYRRECLLKLNEQAAELFESGEDREVNNGLIIMNEFIVPFLPLLLVDEMEEKDILAVEDMRNRWCSYLGQEMEPHLQEKLTDFLPKLLDCSMEIKSFHEPPKLPSYSTHELCERFARIMLSLSRTPADGR",
# UBP26_HUMAN
"MAALFLRGFVQIGNCKTGISKSKEAFIEAVERKKKDRLVLYFKSGKYSTFRLSDNIQNVVLKSYRGNQNHLHLTLQNNNGLFIEGLSSTDAEQLKIFLDRVHQNEVQPPVRPGKGGSVFSSTTQKEINKTSFHKVDEKSSSKSFEIAKGSGTGVLQRMPLLTSKLTLTCGELSENQHKKRKRMLSSSSEMNEEFLKENNSVEYKKSKADCSRCVSYNREKQLKLKELEENKKLECESSCIMNATGNPYLDDIGLLQALTEKMVLVFLLQQGYSDGYTKWDKLKLFFELFPEKICHGLPNLGNTCYMNAVLQSLLSIPSFADDLLNQSFPWGKIPLNALTMCLARLLFFKDTYNIEIKEMLLLNLKKAISAAAEIFHGNAQNDAHEFLAHCLDQLKDNMEKLNTIWKPKSEFGEDNFPKQVFADDPDTSGFSCPVITNFELELLHSIACKACGQVILKTELNNYLSINLPQRIKAHPSSIQSTFDLFFGAEELEYKCAKCEHKTSVGVHSFSRLPRILIVHLKRYSLNEFCALKKNDQEVIISKYLKVSSHCNEGTRPPLPLSEDGEITDFQLLKVIRKMTSGNISVSWPATKESKDILAPHIGSDKESEQKKGQTVFKGASRRQQQKYLGKNSKPNELESVYSGDRAFIEKEPLAHLMTYLEDTSLCQFHKAGGKPASSPGTPLSKVDFQTVPENPKRKKYVKTSKFVAFDRIINPTKDLYEDKNIRIPERFQKVSEQTQQCDGMRICEQAPQQALPQSFPKPGTQGHTKNLLRPTKLNLQKSNRNSLLALGSNKNPRNKDILDKIKSKAKETKRNDDKGDHTYRLISVVSHLGKTLKSGHYICDAYDFEKQIWFTYDDMRVLGIQEAQMQEDRRCTGYIFFYMHNEIFEEMLKREENAQLNSKEVEETLQKE",
# UBP27_HUMAN
"MCKDYVYDKDIEQIAKEEQGEALKLQASTSTEVSHQQCSVPGLGEKFPTWETTKPELELLGHNPRRRRITSSFTIGLRGLINLGNTCFMNCIVQALTHTPILRDFFLSDRHRCEMPSPELCLVCEMSSLFRELYSGNPSPHVPYKLLHLVWIHARHLAGYRQQDAHEFLIAALDVLHRHCKGDDVGKAANNPNHCNCIIDQIFTGGLQSDVTCQACHGVSTTIDPCWDISLDLPGSCTSFWPMSPGRESSVNGESHIPGITTLTDCLRRFTRPEHLGSSAKIKCGSCQSYQESTKQLTMNKLPVVACFHFKRFEHSAKQRRKITTYISFPLELDMTPFMASSKESRMNGQLQLPTNSGNNENKYSLFAVVNHQGTLESGHYTSFIRHHKDQWFKCDDAVITKASIKDVLDSEGYLLFYHKQVLEHESEKVKEMNTQAY",
# UBP28_HUMAN
"MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKASNGDITQAVSLLTDERVKEPSQDTVATEPSEVEGSAANKEVLAKVIDLTHDNKDDLQAAIALSLLESPKIQADGRDLNRMHEATSAETKRSKRKRCEVWGENPNPNDWRRVDGWPVGLKNVGNTCWFSAVIQSLFQLPEFRRLVLSYSLPQNVLENCRSHTEKRNIMFMQELQYLFALMMGSNRKFVDPSAALDLLKGAFRSSEEQQQDVSEFTHKLLDWLEDAFQLAVNVNSPRNKSENPMVQLFYGTFLTEGVREGKPFCNNETFGQYPLQVNGYRNLDECLEGAMVEGDVELLPSDHSVKYGQERWFTKLPPVLTFELSRFEFNQSLGQPEKIHNKLEFPQIIYMDRYMYRSKELIRNKRECIRKLKEEIKILQQKLERYVKYGSGPARFPLPDMLKYVIEFASTKPASESCPPESDTHMTLPLSSVHCSVSDQTSKESTSTESSSQDVESTFSSPEDSLPKSKPLTSSRSSMEMPSQPAPRTVTDEEINFVKTCLQRWRSEIEQDIQDLKTCIASTTQTIEQMYCDPLLRQVPYRLHAVLVHEGQANAGHYWAYIYNQPRQSWLKYNDISVTESSWEEVERDSYGGLRNVSAYCLMYINDKLPYFNAEAAPTESDQMSEVEALSVELKHYIQEDNWRFEQEVEEWEEEQSCKIPQMESSTNSSSQDYSTSQEPSVASSHGVRCLSSEHAVIVKEQTAQAIANTARAYEKSGVEAALSEVMLSPAMQGVILAIAKARQTFDRDGSEAGLIKAFHEEYSRLYQLAKETPTSHSDPRLQHVLVYFFQNEAPKRVVERTLLEQFADKNLSYDERSISIMKVAQAKLKEIGPDDMNMEEYKKWHEDYSLFRKVSVYLLTGLELYQKGKYQEALSYLVYAYQSNAALLMKGPRRGVKESVIALYRRKCLLELNAKAASLFETNDDHSVTEGINVMNELIIPCIHLIINNDISKDDLDAIEVMRNHWCSYLGQDIAENLQLCLGEFLPRLLDPSAEIIVLKEPPTIRPNSPYDLCSRFAAVMESIQGVSTVTVK",
# UBP29_HUMAN
"MISLKVCGFIQIWSQKTGMTKLKEALIETVQRQKEIKLVVTFKSGKFIRIFQLSNNIRSVVLRHCKKRQSHLRLTLKNNVFLFIDKLSYRDAKQLNMFLDIIHQNKSQQPMKSDDDWSVFESRNMLKEIDKTSFYSICNKPSYQKMPLFMSKSPTHVKKGILENQGGKGQNTLSSDVQTNEDILKEDNPVPNKKYKTDSLKYIQSNRKNPSSLEDLEKDRDLKLGPSFNTNCNGNPNLDETVLATQTLNAKNGLTSPLEPEHSQGDPRCNKAQVPLDSHSQQLQQGFPNLGNTCYMNAVLQSLFAIPSFADDLLTQGVPWEYIPFEALIMTLTQLLALKDFCSTKIKRELLGNVKKVISAVAEIFSGNMQNDAHEFLGQCLDQLKEDMEKLNATLNTGKECGDENSSPQMHVGSAATKVFVCPVVANFEFELQLSLICKACGHAVLKVEPNNYLSINLHQETKPLPLSIQNSLDLFFKEEELEYNCQMCKQKSCVARHTFSRLSRVLIIHLKRYSFNNAWLLVKNNEQVYIPKSLSLSSYCNESTKPPLPLSSSAPVGKCEVLEVSQEMISEINSPLTPSMKLTSESSDSLVLPVEPDKNADLQRFQRDCGDASQEQHQRDLENGSALESELVHFRDRAIGEKELPVADSLMDQGDISLPVMYEDGGKLISSPDTRLVEVHLQEVPQHPELQKYEKTNTFVEFNFDSVTESTNGFYDCKENRIPEGSQGMAEQLQQCIEESIIDEFLQQAPPPGVRKLDAQEHTEETLNQSTELRLQKADLNHLGALGSDNPGNKNILDAENTRGEAKELTRNVKMGDPLQAYRLISVVSHIGSSPNSGHYISDVYDFQKQAWFTYNDLCVSEISETKMQEARLHSGYIFFYMHNGIFEELLRKAENSRLPSTQAGVIPQGEYEGDSLYRPA",
# UBP3_HUMAN
"MECPHLSSSVCIAPDSAKFPNGSPSSWCCSVCRSNKSPWVCLTCSSVHCGRYVNGHAKKHYEDAQVPLTNHKKSEKQDKVQHTVCMDCSSYSTYCYRCDDFVVNDTKLGLVQKVREHLQNLENSAFTADRHKKRKLLENSTLNSKLLKVNGSTTAICATGLRNLGNTCFMNAILQSLSNIEQFCCYFKELPAVELRNGKTAGRRTYHTRSQGDNNVSLVEEFRKTLCALWQGSQTAFSPESLFYVVWKIMPNFRGYQQQDAHEFMRYLLDHLHLELQGGFNGVSRSAILQENSTLSASNKCCINGASTVVTAIFGGILQNEVNCLICGTESRKFDPFLDLSLDIPSQFRSKRSKNQENGPVCSLRDCLRSFTDLEELDETELYMCHKCKKKQKSTKKFWIQKLPKVLCLHLKRFHWTAYLRNKVDTYVEFPLRGLDMKCYLLEPENSGPESCLYDLAAVVVHHGSGVGSGHYTAYATHEGRWFHFNDSTVTLTDEETVVKAKAYILFYVEHQAKAGSDKL",
# UBP30_HUMAN
"MLSSRAEAAMTAADRAIQRFLRTGAAVRYKVMKNWGVIGGIAAALAAGIYVIWGPITERKKRRKGLVPGLVNLGNTCFMNSLLQGLSACPAFIRWLEEFTSQYSRDQKEPPSHQYLSLTLLHLLKALSCQEVTDDEVLDASCLLDVLRMYRWQISSFEEQDAHELFHVITSSLEDERDRQPRVTHLFDVHSLEQQSEITPKQITCRTRGSPHPTSNHWKSQHPFHGRLTSNMVCKHCEHQSPVRFDTFDSLSLSIPAATWGHPLTLDHCLHHFISSESVRDVVCDNCTKIEAKGTLNGEKVEHQRTTFVKQLKLGKLPQCLCIHLQRLSWSSHGTPLKRHEHVQFNEFLMMDIYKYHLLGHKPSQHNPKLNKNPGPTLELQDGPGAPTPVLNQPGAPKTQIFMNGACSPSLLPTLSAPMPFPLPVVPDYSSSTYLFRLMAVVVHHGDMHSGHFVTYRRSPPSARNPLSTSNQWLWVSDDTVRKASLQEVLSSSAYLLFYERVLSRMQHQSQECKSEE",
# UBP31_HUMAN
"MSKVTAPGSGPPAAASGKEKRSFSKRLFRSGRAGGGGAGGPGASGPAAPSSPSSPSSARSVGSFMSRVLKTLSTLSHLSSEGAAPDRGGLRSCFPPGPAAAPTPPPCPPPPASPAPPACAAEPVPGVAGLRNHGNTCFMNATLQCLSNTELFAEYLALGQYRAGRPEPSPDPEQPAGRGAQGQGEVTEQLAHLVRALWTLEYTPQHSRDFKTIVSKNALQYRGNSQHDAQEFLLWLLDRVHEDLNHSVKQSGQPPLKPPSETDMMPEGPSFPVCSTFVQELFQAQYRSSLTCPHCQKQSNTFDPFLCISLPIPLPHTRPLYVTVVYQGKCSHCMRIGVAVPLSGTVARLREAVSMETKIPTDQIVLTEMYYDGFHRSFCDTDDLETVHESDCIFAFETPEIFRPEGILSQRGIHLNNNLNHLKFGLDYHRLSSPTQTAAKQGKMDSPTSRAGSDKIVLLVCNRACTGQQGKRFGLPFVLHLEKTIAWDLLQKEILEKMKYFLRPTVCIQVCPFSLRVVSVVGITYLLPQEEQPLCHPIVERALKSCGPGGTAHVKLVVEWDKETRDFLFVNTEDEYIPDAESVRLQRERHHQPQTCTLSQCFQLYTKEERLAPDDAWRCPHCKQLQQGSITLSLWTLPDVLIIHLKRFRQEGDRRMKLQNMVKFPLTGLDMTPHVVKRSQSSWSLPSHWSPWRRPYGLGRDPEDYIYDLYAVCNHHGTMQGGHYTAYCKNSVDGLWYCFDDSDVQQLSEDEVCTQTAYILFYQRRTAIPSWSANSSVAGSTSSSLCEHWVSRLPGSKPASVTSAASSRRTSLASLSESVEMTGERSEDDGGFSTRPFVRSVQRQSLSSRSSVTSPLAVNENCMRPSWSLSAKLQMRSNSPSRFSGDSPIHSSASTLEKIGEAADDKVSISCFGSLRNLSSSYQEPSDSHSRREHKAVGRAPLAVMEGVFKDESDTRRLNSSVVDTQSKHSAQGDRLPPLSGPFDNNNQIAYVDQSDSVDSSPVKEVKAPSHPGSLAKKPESTTKRSPSSKGTSEPEKSLRKGRPALASQESSLSSTSPSSPLPVKVSLKPSRSRSKADSSSRGSGRHSSPAPAQPKKESSPKSQDSVSSPSPQKQKSASALTYTASSTSAKKASGPATRSPFPPGKSRTSDHSLSREGSRQSLGSDRASATSTSKPNSPRVSQARAGEGRGAGKHVRSSSMASLRSPSTSIKSGLKRDSKSEDKGLSFFKSALRQKETRRSTDLGKTALLSKKAGGSSVKSVCKNTGDDEAERGHQPPASQQPNANTTGKEQLVTKDPASAKHSLLSARKSKSSQLDSGVPSSPGGRQSAEKSSKKLSSSMQTSARPSQKPQ",
# UBP32_HUMAN
"MGAKESRIGFLSYEEALRRVTDVELKRLKDAFKRTCGLSYYMGQHCFIREVLGDGVPPKVAEVIYCSFGGTSKGLHFNNLIVGLVLLTRGKDEEKAKYIFSLFSSESGNYVIREEMERMLHVVDGKVPDTLRKCFSEGEKVNYEKFRNWLFLNKDAFTFSRWLLSGGVYVTLTDDSDTPTFYQTLAGVTHLEESDIIDLEKRYWLLKAQSRTGRFDLETFGPLVSPPIRPSLSEGLFNAFDENRDNHIDFKEISCGLSACCRGPLAERQKFCFKVFDVDRDGVLSRVELRDMVVALLEVWKDNRTDDIPELHMDLSDIVEGILNAHDTTKMGHLTLEDYQIWSVKNVLANEFLNLLFQVCHIVLGLRPATPEEEGQIIRGWLERESRYGLQAGHNWFIISMQWWQQWKEYVKYDANPVVIEPSSVLNGGKYSFGTAAHPMEQVEDRIGSSLSYVNTTEEKFSDNISTASEASETAGSGFLYSATPGADVCFARQHNTSDNNNQCLLGANGNILLHLNPQKPGAIDNQPLVTQEPVKATSLTLEGGRLKRTPQLIHGRDYEMVPEPVWRALYHWYGANLALPRPVIKNSKTDIPELELFPRYLLFLRQQPATRTQQSNIWVNMGNVPSPNAPLKRVLAYTGCFSRMQTIKEIHEYLSQRLRIKEEDMRLWLYNSENYLTLLDDEDHKLEYLKIQDEQHLVIEVRNKDMSWPEEMSFIANSSKIDRHKVPTEKGATGLSNLGNTCFMNSSIQCVSNTQPLTQYFISGRHLYELNRTNPIGMKGHMAKCYGDLVQELWSGTQKNVAPLKLRWTIAKYAPRFNGFQQQDSQELLAFLLDGLHEDLNRVHEKPYVELKDSDGRPDWEVAAEAWDNHLRRNRSIVVDLFHGQLRSQVKCKTCGHISVRFDPFNFLSLPLPMDSYMHLEITVIKLDGTTPVRYGLRLNMDEKYTGLKKQLSDLCGLNSEQILLAEVHGSNIKNFPQDNQKVRLSVSGFLCAFEIPVPVSPISASSPTQTDFSSSPSTNEMFTLTTNGDLPRPIFIPNGMPNTVVPCGTEKNFTNGMVNGHMPSLPDSPFTGYIIAVHRKMMRTELYFLSSQKNRPSLFGMPLIVPCTVHTRKKDLYDAVWIQVSRLASPLPPQEASNHAQDCDDSMGYQYPFTLRVVQKDGNSCAWCPWYRFCRGCKIDCGEDRAFIGNAYIAVDWDPTALHLRYQTSQERVVDEHESVEQSRRAQAEPINLDSCLRAFTSEEELGENEMYYCSKCKTHCLATKKLDLWRLPPILIIHLKRFQFVNGRWIKSQKIVKFPRESFDPSAFLVPRDPALCQHKPLTPQGDELSEPRILAREVKKVDAQSSAGEEDVLLSKSPSSLSANIISSPKGSPSSSRKSGTSCPSSKNSSPNSSPRTLGRSKGRLRLPQIGSKNKLSSSKENLDASKENGAGQICELADALSRGHVLGGSQPELVTPQDHEVALANGFLYEHEACGNGYSNGQLGNHSEEDSTDDQREDTRIKPIYNLYAISCHSGILGGGHYVTYAKNPNCKWYCYNDSSCKELHPDEIDTDSAYILFYEQQGIDYAQFLPKTDGKKMADTSSMDEDFESDYKKYCVLQ",
# UBP33_HUMAN
"MTGSNSHITILTLKVLPHFESLGKQEKIPNKMSAFRNHCPHLDSVGEITKEDLIQKSLGTCQDCKVQGPNLWACLENRCSYVGCGESQVDHSTIHSQETKHYLTVNLTTLRVWCYACSKEVFLDRKLGTQPSLPHVRQPHQIQENSVQDFKIPSNTTLKTPLVAVFDDLDIEADEEDELRARGLTGLKNIGNTCYMNAALQALSNCPPLTQFFLDCGGLARTDKKPAICKSYLKLMTELWHKSRPGSVVPTTLFQGIKTVNPTFRGYSQQDAQEFLRCLMDLLHEELKEQVMEVEEDPQTITTEETMEEDKSQSDVDFQSCESCSNSDRAENENGSRCFSEDNNETTMLIQDDENNSEMSKDWQKEKMCNKINKVNSEGEFDKDRDSISETVDLNNQETVKVQIHSRASEYITDVHSNDLSTPQILPSNEGVNPRLSASPPKSGNLWPGLAPPHKKAQSASPKRKKQHKKYRSVISDIFDGTIISSVQCLTCDRVSVTLETFQDLSLPIPGKEDLAKLHSSSHPTSIVKAGSCGEAYAPQGWIAFFMEYVKRFVVSCVPSWFWGPVVTLQDCLAAFFARDELKGDNMYSCEKCKKLRNGVKFCKVQNFPEILCIHLKRFRHELMFSTKISTHVSFPLEGLDLQPFLAKDSPAQIVTYDLLSVICHHGTASSGHYIAYCRNNLNNLWYEFDDQSVTEVSESTVQNAEAYVLFYRKSSEEAQKERRRISNLLNIMEPSLLQFYISRQWLNKFKTFAEPGPISNNDFLCIHGGVPPRKAGYIEDLVLMLPQNIWDNLYSRYGGGPAVNHLYICHTCQIEAEKIEKRRKTELEIFIRLNRAFQKEDSPATFYCISMQWFREWESFVKGKDGDPPGPIDNTKIAVTKCGNVMLRQGADSGQISEETWNFLQSIYGGGPEVILRPPVVHVDPDILQAEEKIEVETRSL",
# UBP34_HUMAN
"MCENCADLVEVLNEISDVEGGDGLQLRKEHTLKIFTYINSWTQRQCLCCFKEYKHLEIFNQVVCALINLVIAQVQVLRDQLCKHCTTINIDSTWQDESNQAEEPLNIDRECNEGSTERQKSIEKKSNSTRICNLTEEESSKSSDPFSLWSTDEKEKLLLCVAKIFQIQFPLYTAYKHNTHPTIEDISTQESNILGAFCDMNDVEVPLHLLRYVCLFCGKNGLSLMKDCFEYGTPETLPFLIAHAFITVVSNIRIWLHIPAVMQHIIPFRTYVIRYLCKLSDQELRQSAARNMADLMWSTVKEPLDTTLCFDKESLDLAFKYFMSPTLTMRLAGLSQITNQLHTFNDVCNNESLVSDTETSIAKELADWLISNNVVEHIFGPNLHIEIIKQCQVILNFLAAEGRLSTQHIDCIWAAAQLKHCSRYIHDLFPSLIKNLDPVPLRHLLNLVSALEPSVHTEQTLYLASMLIKALWNNALAAKAQLSKQSSFASLLNTNIPIGNKKEEEELRRTAPSPWSPAASPQSSDNSDTHQSGGSDIEMDEQLINRTKHVQQRLSDTEESMQGSSDETANSGEDGSSGPGSSSGHSDGSSNEVNSSHASQSAGSPGSEVQSEDIADIEALKEEDEDDDHGHNPPKSSCGTDLRNRKLESQAGICLGDSQGMSERNGTSSGTGKDLVFNTESLPSVDNRMRMLDACSHSEDPEHDISGEMNATHIAQGSQESCITRTGDFLGETIGNELFNCRQFIGPQHHHHHHHHHHHHDGHMVDDMLSADDVSCSSSQVSAKSEKNMADFDGEESGCEEELVQINSHAELTSHLQQHLPNLASIYHEHLSQGPVVHKHQFNSNAVTDINLDNVCKKGNTLLWDIVQDEDAVNLSEGLINEAEKLLCSLVCWFTDRQIRMRFIEGCLENLGNNRSVVISLRLLPKLFGTFQQFGSSYDTHWITMWAEKELNMMKLFFDNLVYYIQTVREGRQKHALYSHSAEVQVRLQFLTCVFSTLGSPDHFRLSLEQVDILWHCLVEDSECYDDALHWFLNQVRSKDQHAMGMETYKHLFLEKMPQLKPETISMTGLNLFQHLCNLARLATSAYDGCSNSELCGMDQFWGIALRAQSGDVSRAAIQYINSYYINGKTGLEKEQEFISKCMESLMIASSSLEQESHSSLMVIERGLLMLKTHLEAFRRRFAYHLRQWQIEGTGISSHLKALSDKQSLPLRVVCQPAGLPDKMTIEMYPSDQVADLRAEVTHWYENLQKEQINQQAQLQEFGQSNRKGEFPGGLMGPVRMISSGHELTTDYDEKALHELGFKDMQMVFVSLGAPRRERKGEGVQLPASCLPPPQKDNIPMLLLLQEPHLTTLFDLLEMLASFKPPSGKVAVDDSESLRCEELHLHAENLSRRVWELLMLLPTCPNMLMAFQNISDEQSNDGFNWKELLKIKSAHKLLYALEIIEALGKPNRRIRRESTGSYSDLYPDSDDSSEDQVENSKNSWSCKFVAAGGLQQLLEIFNSGILEPKEQESWTVWQLDCLACLLKLICQFAVDPSDLDLAYHDVFAWSGIAESHRKRTWPGKSRKAAGDHAKGLHIPRLTEVFLVLVQGTSLIQRLMSVAYTYDNLAPRVLKAQSDHRSRHEVSHYSMWLLVSWAHCCSLVKSSLADSDHLQDWLKKLTLLIPETAVRHESCSGLYKLSLSGLDGGDSINRSFLLLAASTLLKFLPDAQALKPIRIDDYEEEPILKPGCKEYFWLLCKLVDNIHIKDASQTTLLDLDALARHLADCIRSREILDHQDGNVEDDGLTGLLRLATSVVKHKPPFKFSREGQEFLRDIFNLLFLLPSLKDRQQPKCKSHSSRAAAYDLLVEMVKGSVENYRLIHNWVMAQHMQSHAPYKWDYWPHEDVRAECRFVGLTNLGATCYLASTIQQLYMIPEARQAVFTAKYSEDMKHKTTLLELQKMFTYLMESECKAYNPRPFCKTYTMDKQPLNTGEQKDMTEFFTDLITKIEEMSPELKNTVKSLFGGVITNNVVSLDCEHVSQTAEEFYTVRCQVADMKNIYESLDEVTIKDTLEGDNMYTCSHCGKKVRAEKRACFKKLPRILSFNTMRYTFNMVTMMKEKVNTHFSFPLRLDMTPYTEDFLMGKSERKEGFKEVSDHSKDSESYEYDLIGVTVHTGTADGGHYYSFIRDIVNPHAYKNNKWYLFNDAEVKPFDSAQLASECFGGEMTTKTYDSVTDKFMDFSFEKTHSAYMLFYKRMEPEEENGREYKFDVSSELLEWIWHDNMQFLQDKNIFEHTYFGFMWQLCSCIPSTLPDPKAVSLMTAKLSTSFVLETFIHSKEKPTMLQWIELLTKQFNNSQAACEWFLDRMADDDWWPMQILIKCPNQIVRQMFQRLCIHVIQRLRPVHAHLYLQPGMEDGSDDMDTSVEDIGGRSCVTRFVRTLLLIMEHGVKPHSKHLTEYFAFLYEFAKMGEEESQFLLSLQAISTMVHFYMGTKGPENPQVEVLSEEEGEEEEEEEDILSLAEEKYRPAALEKMIALVALLVEQSRSERHLTLSQTDMAALTGGKGFPFLFQHIRDGINIRQTCNLIFSLCRYNNRLAEHIVSMLFTSIAKLTPEAANPFFKLLTMLMEFAGGPPGMPPFASYILQRIWEVIEYNPSQCLDWLAVQTPRNKLAHSWVLQNMENWVERFLLAHNYPRVRTSAAYLLVSLIPSNSFRQMFRSTRSLHIPTRDLPLSPDTTVVLHQVYNVLLGLLSRAKLYVDAAVHGTTKLVPYFSFMTYCLISKTEKLMFSTYFMDLWNLFQPKLSEPAIATNHNKQALLSFWYNVCADCPENIRLIVQNPVVTKNIAFNYILADHDDQDVVLFNRGMLPAYYGILRLCCEQSPAFTRQLASHQNIQWAFKNLTPHASQYPGAVEELFNLMQLFIAQRPDMREEELEDIKQFKKTTISCYLRCLDGRSCWTTLISAFRILLESDEDRLLVVFNRGLILMTESFNTLHMMYHEATACHVTGDLVELLSIFLSVLKSTRPYLQRKDVKQALIQWQERIEFAHKLLTLLNSYSPPELRNACIDVLKELVLLSPHDFLHTLVPFLQHNHCTYHHSNIPMSLGPYFPCRENIKLIGGKSNIRPPRPELNMCLLPTMVETSKGKDDVYDRMLLDYFFSYHQFIHLLCRVAINCEKFTETLVKLSVLVAYEGLPLHLALFPKLWTELCQTQSAMSKNCIKLLCEDPVFAEYIKCILMDERTFLNNNIVYTFMTHFLLKVQSQVFSEANCANLISTLITNLISQYQNLQSDFSNRVEISKASASLNGDLRALALLLSVHTPKQLNPALIPTLQELLSKCRTCLQQRNSLQEQEAKERKTKDDEGATPIKRRRVSSDEEHTVDSCISDMKTETREVLTPTSTSDNETRDSSIIDPGTEQDLPSPENSSVKEYRMEVPSSFSEDMSNIRSQHAEEQSNNGRYDDCKEFKDLHCSKDSTLAEEESEFPSTSISAVLSDLADLRSCDGQALPSQDPEVALSLSCGHSRGLFSHMQQHDILDTLCRTIESTIHVVTRISGKGNQAAS",
# UBP35_HUMAN
"MDKILEAVVTSSYPVSVKQGLVRRVLEAARQPLEREQCLALLALGARLYVGGAEELPRRVGCQLLHVAGRHHPDVFAEFFSARRVLRLLQGGAGPPGPRALACVQLGLQLLPEGPAADEVFALLRREVLRTVCERPGPAACAQVARLLARHPRCVPDGPHRLLFCQQLVRCLGRFRCPAEGEEGAVEFLEQAQQVSGLLAQLWRAQPAAILPCLKELFAVISCAEEEPPSSALASVVQHLPLELMDGVVRNLSNDDSVTDSQMLTAISRMIDWVSWPLGKNIDKWIIALLKGLAAVKKFSILIEVSLTKIEKVFSKLLYPIVRGAALSVLKYMLLTFQHSHEAFHLLLPHIPPMVASLVKEDSNSGTSCLEQLAELVHCMVFRFPGFPDLYEPVMEAIKDLHVPNEDRIKQLLGQDAWTSQKSELAGFYPRLMAKSDTGKIGLINLGNTCYVNSILQALFMASDFRHCVLRLTENNSQPLMTKLQWLFGFLEHSQRPAISPENFLSASWTPWFSPGTQQDCSEYLKYLLDRLHEEEKTGTRICQKLKQSSSPSPPEEPPAPSSTSVEKMFGGKIVTRICCLCCLNVSSREEAFTDLSLAFPPPERCRRRRLGSVMRPTEDITARELPPPTSAQGPGRVGPRRQRKHCITEDTPPTSLYIEGLDSKEAGGQSSQEERIEREEEGKEERTEKEEVGEEEESTRGEGEREKEEEVEEEEEKVEKETEKEAEQEKEEDSLGAGTHPDAAIPSGERTCGSEGSRSVLDLVNYFLSPEKLTAENRYYCESCASLQDAEKVVELSQGPCYLILTLLRFSFDLRTMRRRKILDDVSIPLLLRLPLAGGRGQAYDLCSVVVHSGVSSESGHYYCYAREGAARPAASLGTADRPEPENQWYLFNDTRVSFSSFESVSNVTSFFPKDTAYVLFYRQRPREGPEAELGSSRVRTEPTLHKDLMEAISKDNILYLQEQEKEARSRAAYISALPTSPHWGRGFDEDKDEDEGSPGGCNPAGGNGGDFHRLVF",
# UBP36_HUMAN
"MPIVDKLKEALKPGRKDSADDGELGKLLASSAKKVLLQKIEFEPASKSFSYQLEALKSKYVLLNPKTEGASRHKSGDDPPARRQGSEHTYESCGDGVPAPQKVLFPTERLSLRWERVFRVGAGLHNLGNTCFLNATIQCLTYTPPLANYLLSKEHARSCHQGSFCMLCVMQNHIVQAFANSGNAIKPVSFIRDLKKIARHFRFGNQEDAHEFLRYTIDAMQKACLNGCAKLDRQTQATTLVHQIFGGYLRSRVKCSVCKSVSDTYDPYLDVALEIRQAANIVRALELFVKADVLSGENAYMCAKCKKKVPASKRFTIHRTSNVLTLSLKRFANFSGGKITKDVGYPEFLNIRPYMSQNNGDPVMYGLYAVLVHSGYSCHAGHYYCYVKASNGQWYQMNDSLVHSSNVKVVLNQQAYVLFYLRIPGSKKSPEGLISRTGSSSLPGRPSVIPDHSKKNIGNGIISSPLTGKRQDSGTMKKPHTTEEIGVPISRNGSTLGLKSQNGCIPPKLPSGSPSPKLSQTPTHMPTILDDPGKKVKKPAPPQHFSPRTAQGLPGTSNSNSSRSGSQRQGSWDSRDVVLSTSPKLLATATANGHGLKGNDESAGLDRRGSSSSSPEHSASSDSTKAPQTPRSGAAHLCDSQETNCSTAGHSKTPPSGADSKTVKLKSPVLSNTTTEPASTMSPPPAKKLALSAKKASTLWRATGNDLRPPPPSPSSDLTHPMKTSHPVVASTWPVHRARAVSPAPQSSSRLQPPFSPHPTLLSSTPKPPGTSEPRSCSSISTALPQVNEDLVSLPHQLPEASEPPQSPSEKRKKTFVGEPQRLGSETRLPQHIREATAAPHGKRKRKKKKRPEDTAASALQEGQTQRQPGSPMYRREGQAQLPAVRRQEDGTQPQVNGQQVGCVTDGHHASSRKRRRKGAEGLGEEGGLHQDPLRHSCSPMGDGDPEAMEESPRKKKKKKRKQETQRAVEEDGHLKCPRSAKPQDAVVPESSSCAPSANGWCPGDRMGLSQAPPVSWNGERESDVVQELLKYSSDKAYGRKVLTWDGKMSAVSQDAIEDSRQARTETVVDDWDEEFDRGKEKKIKKFKREKRRNFNAFQKLQTRRNFWSVTHPAKAASLSYRR",
# UBP37_HUMAN
"MSPLKIHGPIRIRSMQTGITKWKEGSFEIVEKENKVSLVVHYNTGGIPRIFQLSHNIKNVVLRPSGAKQSRLMLTLQDNSFLSIDKVPSKDAEEMRLFLDAVHQNRLPAAMKPSQGSGSFGAILGSRTSQKETSRQLSYSDNQASAKRGSLETKDDIPFRKVLGNPGRGSIKTVAGSGIARTIPSLTSTSTPLRSGLLENRTEKRKRMISTGSELNEDYPKENDSSSNNKAMTDPSRKYLTSSREKQLSLKQSEENRTSGLLPLQSSSFYGSRAGSKEHSSGGTNLDRTNVSSQTPSAKRSLGFLPQPVPLSVKKLRCNQDYTGWNKPRVPLSSHQQQQLQGFSNLGNTCYMNAILQSLFSLQSFANDLLKQGIPWKKIPLNALIRRFAHLLVKKDICNSETKKDLLKKVKNAISATAERFSGYMQNDAHEFLSQCLDQLKEDMEKLNKTWKTEPVSGEENSPDISATRAYTCPVITNLEFEVQHSIICKACGEIIPKREQFNDLSIDLPRRKKPLPPRSIQDSLDLFFRAEELEYSCEKCGGKCALVRHKFNRLPRVLILHLKRYSFNVALSLNNKIGQQVIIPRYLTLSSHCTENTKPPFTLGWSAHMAISRPLKASQMVNSCITSPSTPSKKFTFKSKSSLALCLDSDSEDELKRSVALSQRLCEMLGNEQQQEDLEKDSKLCPIEPDKSELENSGFDRMSEEELLAAVLEISKRDASPSLSHEDDDKPTSSPDTGFAEDDIQEMPENPDTMETEKPKTITELDPASFTEITKDCDENKENKTPEGSQGEVDWLQQYDMEREREEQELQQALAQSLQEQEAWEQKEDDDLKRATELSLQEFNNSFVDALGSDEDSGNEDVFDMEYTEAEAEELKRNAETGNLPHSYRLISVVSHIGSTSSSGHYISDVYDIKKQAWFTYNDLEVSKIQEAAVQSDRDRSGYIFFYMHKEIFDELLETEKNSQSLSTEVGKTTRQAL",
# UBP38_HUMAN
"MDKILEGLVSSSHPLPLKRVIVRKVVESAEHWLDEAQCEAMFDLTTRLILEGQDPFQRQVGHQVLEAYARYHRPEFESFFNKTFVLGLLHQGYHSLDRKDVAILDYIHNGLKLIMSCPSVLDLFSLLQVEVLRMVCERPEPQLCARLSDLLTDFVQCIPKGKLSITFCQQLVRTIGHFQCVSTQERELREYVSQVTKVSNLLQNIWKAEPATLLPSLQEVFASISSTDASFEPSVALASLVQHIPLQMITVLIRSLTTDPNVKDASMTQALCRMIDWLSWPLAQHVDTWVIALLKGLAAVQKFTILIDVTLLKIELVFNRLWFPLVRPGALAVLSHMLLSFQHSPEAFHLIVPHVVNLVHSFKNDGLPSSTAFLVQLTELIHCMMYHYSGFPDLYEPILEAIKDFPKPSEEKIKLILNQSAWTSQSNSLASCLSRLSGKSETGKTGLINLGNTCYMNSVIQALFMATDFRRQVLSLNLNGCNSLMKKLQHLFAFLAHTQREAYAPRIFFEASRPPWFTPRSQQDCSEYLRFLLDRLHEEEKILKVQASHKPSEILECSETSLQEVASKAAVLTETPRTSDGEKTLIEKMFGGKLRTHIRCLNCRSTSQKVEAFTDLSLAFCPSSSLENMSVQDPASSPSIQDGGLMQASVPGPSEEPVVYNPTTAAFICDSLVNEKTIGSPPNEFYCSENTSVPNESNKILVNKDVPQKPGGETTPSVTDLLNYFLAPEILTGDNQYYCENCASLQNAEKTMQITEEPEYLILTLLRFSYDQKYHVRRKILDNVSLPLVLELPVKRITSFSSLSESWSVDVDFTDLSENLAKKLKPSGTDEASCTKLVPYLLSSVVVHSGISSESGHYYSYARNITSTDSSYQMYHQSEALALASSQSHLLGRDSPSAVFEQDLENKEMSKEWFLFNDSRVTFTSFQSVQKITSRFPKDTAYVLLYKKQHSTNGLSGNNPTSGLWINGDPPLQKELMDAITKDNKLYLQEQELNARARALQAASASCSFRPNGFDDNDPPGSCGPTGGGGGGGFNTVGRLVF",
# UBP39_HUMAN
"MSGRSKRESRGSTRGKRESESRGSSGRVKRERDREREPEAASSRGSPVRVKREFEPASAREAPASVVPFVRVKREREVDEDSEPEREVRAKNGRVDSEDRRSRHCPYLDTINRSVLDFDFEKLCSISLSHINAYACLVCGKYFQGRGLKSHAYIHSVQFSHHVFLNLHTLKFYCLPDNYEIIDSSLEDITYVLKPTFTKQQIANLDKQAKLSRAYDGTTYLPGIVGLNNIKANDYANAVLQALSNVPPLRNYFLEEDNYKNIKRPPGDIMFLLVQRFGELMRKLWNPRNFKAHVSPHEMLQAVVLCSKKTFQITKQGDGVDFLSWFLNALHSALGGTKKKKKTIVTDVFQGSMRIFTKKLPHPDLPAEEKEQLLHNDEYQETMVESTFMYLTLDLPTAPLYKDEKEQLIIPQVPLFNILAKFNGITEKEYKTYKENFLKRFQLTKLPPYLIFCIKRFTKNNFFVEKNPTIVNFPITNVDLREYLSEEVQAVHKNTTYDLIANIVHDGKPSEGSYRIHVLHHGTGKWYELQDLQVTDILPQMITLSEAYIQIWKRRDNDETNQQGA",
# UBP4_HUMAN
"MAEGGGCRERPDAETQKSELGPLMRTTLQRGAQWYLIDSRWFKQWKKYVGFDSWDMYNVGEHNLFPGPIDNSGLFSDPESQTLKEHLIDELDYVLVPTEAWNKLLNWYGCVEGQQPIVRKVVEHGLFVKHCKVEVYLLELKLCENSDPTNVLSCHFSKADTIATIEKEMRKLFNIPAERETRLWNKYMSNTYEQLSKLDNTVQDAGLYQGQVLVIEPQNEDGTWPRQTLQSKSSTAPSRNFTTSPKSSASPYSSVSASLIANGDSTSTCGMHSSGVSRGGSGFSASYNCQEPPSSHIQPGLCGLGNLGNTCFMNSALQCLSNTAPLTDYFLKDEYEAEINRDNPLGMKGEIAEAYAELIKQMWSGRDAHVAPRMFKTQVGRFAPQFSGYQQQDSQELLAFLLDGLHEDLNRVKKKPYLELKDANGRPDAVVAKEAWENHRLRNDSVIVDTFHGLFKSTLVCPECAKVSVTFDPFCYLTLPLPLKKDRVMEVFLVPADPHCRPTQYRVTVPLMGAVSDLCEALSRLSGIAAENMVVADVYNHRFHKIFQMDEGLNHIMPRDDIFVYEVCSTSVDGSECVTLPVYFRERKSRPSSTSSASALYGQPLLLSVPKHKLTLESLYQAVCDRISRYVKQPLPDEFGSSPLEPGACNGSRNSCEGEDEEEMEHQEEGKEQLSETEGSGEDEPGNDPSETTQKKIKGQPCPKRLFTFSLVNSYGTADINSLAADGKLLKLNSRSTLAMDWDSETRRLYYDEQESEAYEKHVSMLQPQKKKKTTVALRDCIELFTTMETLGEHDPWYCPNCKKHQQATKKFDLWSLPKILVVHLKRFSYNRYWRDKLDTVVEFPIRGLNMSEFVCNLSARPYVYDLIAVSNHYGAMGVGHYTAYAKNKLNGKWYYFDDSNVSLASEDQIVTKAAYVLFYQRRDDEFYKTPSLSSSGSSDGGTRPSSSQQGFGDDEACSMDTN",
# UBP40_HUMAN
"MFGDLFEEEYSTVSNNQYGKGKKLKTKALEPPAPREFTNLSGIRNQGGTCYLNSLLQTLHFTPEFREALFSLGPEELGLFEDKDKPDAKVRIIPLQLQRLFAQLLLLDQEAASTADLTDSFGWTSNEEMRQHDVQELNRILFSALETSLVGTSGHDLIYRLYHGTIVNQIVCKECKNVSERQEDFLDLTVAVKNVSGLEDALWNMYVEEEVFDCDNLYHCGTCDRLVKAAKSAKLRKLPPFLTVSLLRFNFDFVKCERYKETSCYTFPLRINLKPFCEQSELDDLEYIYDLFSVIIHKGGCYGGHYHVYIKDVDHLGNWQFQEEKSKPDVNLKDLQSEEEIDHPLMILKAILLEENNLIPVDQLGQKLLKKIGISWNKKYRKQHGPLRKFLQLHSQIFLLSSDESTVRLLKNSSLQAESDFQRNDQQIFKMLPPESPGLNNSISCPHWFDINDSKVQPIREKDIEQQFQGKESAYMLFYRKSQLQRPPEARANPRYGVPCHLLNEMDAANIELQTKRAECDSANNTFELHLHLGPQYHFFNGALHPVVSQTESVWDLTFDKRKTLGDLRQSIFQLLEFWEGDMVLSVAKLVPAGLHIYQSLGGDELTLCETEIADGEDIFVWNGVEVGGVHIQTGIDCEPLLLNVLHLDTSSDGEKCCQVIESPHVFPANAEVGTVLTALAIPAGVIFINSAGCPGGEGWTAIPKEDMRKTFREQGLRNGSSILIQDSHDDNSLLTKEEKWVTSMNEIDWLHVKNLCQLESEEKQVKISATVNTMVFDIRIKAIKELKLMKELADNSCLRPIDRNGKLLCPVPDSYTLKEAELKMGSSLGLCLGKAPSSSQLFLFFAMGSDVQPGTEMEIVVEETISVRDCLKLMLKKSGLQGDAWHLRKMDWCYEAGEPLCEEDATLKELLICSGDTLLLIEGQLPPLGFLKVPIWWYQLQGPSGHWESHQDQTNCTSSWGRVWRATSSQGASGNEPAQVSLLYLGDIEISEDATLAELKSQAMTLPPFLEFGVPSPAHLRAWTVERKRPGRLLRTDRQPLREYKLGRRIEICLEPLQKGENLGPQDVLLRTQVRIPGERTYAPALDLVWNAAQGGTAGSLRQRVADFYRLPVEKIEIAKYFPEKFEWLPISSWNQQITKRKKKKKQDYLQGAPYYLKDGDTIGVKNLLIDDDDDFSTIRDDTGKEKQKQRALGRRKSQEALHEQSSYILSSAETPARPRAPETSLSIHVGSFR",
# UBP42_HUMAN
"MTIVDKASESSDPSAYQNQPGSSEAVSPGDMDAGSASWGAVSSLNDVSNHTLSLGPVPGAVVYSSSSVPDKSKPSPQKDQALGDGIAPPQKVLFPSEKICLKWQQTHRVGAGLQNLGNTCFANAALQCLTYTPPLANYMLSHEHSKTCHAEGFCMMCTMQAHITQALSNPGDVIKPMFVINEMRRIARHFRFGNQEDAHEFLQYTVDAMQKACLNGSNKLDRHTQATTLVCQIFGGYLRSRVKCLNCKGVSDTFDPYLDITLEIKAAQSVNKALEQFVKPEQLDGENSYKCSKCKKMVPASKRFTIHRSSNVLTLSLKRFANFTGGKIAKDVKYPEYLDIRPYMSQPNGEPIVYVLYAVLVHTGFNCHAGHYFCYIKASNGLWYQMNDSIVSTSDIRSVLSQQAYVLFYIRSHDVKNGGELTHPTHSPGQSSPRPVISQRVVTNKQAAPGFIGPQLPSHMIKNPPHLNGTGPLKDTPSSSMSSPNGNSSVNRASPVNASASVQNWSVNRSSVIPEHPKKQKITISIHNKLPVRQCQSQPNLHSNSLENPTKPVPSSTITNSAVQSTSNASTMSVSSKVTKPIPRSESCSQPVMNGKSKLNSSVLVPYGAESSEDSDEESKGLGKENGIGTIVSSHSPGQDAEDEEATPHELQEPMTLNGANSADSDSDPKENGLAPDGASCQGQPALHSENPFAKANGLPGKLMPAPLLSLPEDKILETFRLSNKLKGSTDEMSAPGAERGPPEDRDAEPQPGSPAAESLEEPDAAAGLSSTKKAPPPRDPGTPATKEGAWEAMAVAPEEPPPSAGEDIVGDTAPPDLCDPGSLTGDASPLSQDAKGMIAEGPRDSALAEAPEGLSPAPPARSEEPCEQPLLVHPSGDHARDAQDPSQSLGAPEAAERPPAPVLDMAPAGHPEGDAEPSPGERVEDAAAPKAPGPSPAKEKIGSLRKVDRGHYRSRRERSSSGEPARESRSKTEGHRHRRRRTCPRERDRQDRHAPEHHPGHGDRLSPGERRSLGRCSHHHSRHRSGVELDWVRHHYTEGERGWGREKFYPDRPRWDRCRYYHDRYALYAARDWKPFHGGREHERAGLHERPHKDHNRGRRGCEPARERERHRPSSPRAGAPHALAPHPDRFSHDRTALVAGDNCNLSDRFHEHENGKSRKRRHDSVENSDSHVEKKARRSEQKDPLEEPKAKKHKKSKKKKKSKDKHRDRDSRHQQDSDLSAACSDADLHRHKKKKKKKKRHSRKSEDFVKDSELHLPRVTSLETVAQFRRAQGGFPLSGGPPLEGVGPFREKTKHLRMESRDDRCRLFEYGQGKRRYLELGR",
# UBP43_HUMAN
"MDLGPGDAAGGGPLAPRPRRRRSLRRLFSRFLLALGSRSRPGDSPPRPQPGHCDGDGEGGFACAPGPVPAAPGSPGEERPPGPQPQLQLPAGDGARPPGAQGLKNHGNTCFMNAVVQCLSNTDLLAEFLALGRYRAAPGRAEVTEQLAALVRALWTREYTPQLSAEFKNAVSKYGSQFQGNSQHDALEFLLWLLDRVHEDLEGSSRGPVSEKLPPEATKTSENCLSPSAQLPLGQSFVQSHFQAQYRSSLTCPHCLKQSNTFDPFLCVSLPIPLRQTRFLSVTLVFPSKSQRFLRVGLAVPILSTVAALRKMVAEEGGVPADEVILVELYPSGFQRSFFDEEDLNTIAEGDNVYAFQVPPSPSQGTLSAHPLGLSASPRLAAREGQRFSLSLHSESKVLILFCNLVGSGQQASRFGPPFLIREDRAVSWAQLQQSILSKVRHLMKSEAPVQNLGSLFSIRVVGLSVACSYLSPKDSRPLCHWAVDRVLHLRRPGGPPHVKLAVEWDSSVKERLFGSLQEERAQDADSVWQQQQAHQQHSCTLDECFQFYTKEEQLAQDDAWKCPHCQVLQQGMVKLSLWTLPDILIIHLKRFCQVGERRNKLSTLVKFPLSGLNMAPHVAQRSTSPEAGLGPWPSWKQPDCLPTSYPLDFLYDLYAVCNHHGNLQGGHYTAYCRNSLDGQWYSYDDSTVEPLREDEVNTRGAYILFYQKRNSIPPWSASSSMRGSTSSSLSDHWLLRLGSHAGSTRGSLLSWSSAPCPSLPQVPDSPIFTNSLCNQEKGGLEPRRLVRGVKGRSISMKAPTTSRAKQGPFKTMPLRWSFGSKEKPPGASVELVEYLESRRRPRSTSQSIVSLLTGTAGEDEKSASPRSNVALPANSEDGGRAIERGPAGVPCPSAQPNHCLAPGNSDGPNTARKLKENAGQDIKLPRKFDLPLTVMPSVEHEKPARPEGQKAMNWKESFQMGSKSSPPSPYMGFSGNSKDSRRGTSELDRPLQGTLTLLRSVFRKKENRRNERAEVSPQVPPVSLVSGGLSPAMDGQAPGSPPALRIPEGLARGLGSRLERDVWSAPSSLRLPRKASRAPRGSALGMSQRTVPGEQASYGTFQRVKYHTLSLGRKKTLPESSF",
# UBP44_HUMAN
"MLAMDTCKHVGQLQLAQDHSSLNPQKWHCVDCNTTESIWACLSCSHVACGRYIEEHALKHFQESSHPVALEVNEMYVFCYLCDDYVLNDNTTGDLKLLRRTLSAIKSQNYHCTTRSGRFLRSMGTGDDSYFLHDGAQSLLQSEDQLYTALWHRRRILMGKIFRTWFEQSPIGRKKQEEPFQEKIVVKREVKKRRQELEYQVKAELESMPPRKSLRLQGLAQSTIIEIVSVQVPAQTPASPAKDKVLSTSENEISQKVSDSSVKRRPIVTPGVTGLRNLGNTCYMNSVLQVLSHLLIFRQCFLKLDLNQWLAMTASEKTRSCKHPPVTDTVVYQMNECQEKDTGFVCSRQSSLSSGLSGGASKGRKMELIQPKEPTSQYISLCHELHTLFQVMWSGKWALVSPFAMLHSVWRLIPAFRGYAQQDAQEFLCELLDKIQRELETTGTSLPALIPTSQRKLIKQVLNVVNNIFHGQLLSQVTCLACDNKSNTIEPFWDLSLEFPERYQCSGKDIASQPCLVTEMLAKFTETEALEGKIYVCDQCNSKRRRFSSKPVVLTEAQKQLMICHLPQVLRLHLKRFRWSGRNNREKIGVHVGFEEILNMEPYCCRETLKSLRPECFIYDLSAVVMHHGKGFGSGHYTAYCYNSEGGFWVHCNDSKLSMCTMDEVCKAQAYILFYTQRVTENGHSKLLPPELLLGSQHPNEDADTSSNEILS",
# UBP45_HUMAN
"MRVKDPTKALPEKAKRSKRPTVPHDEDSSDDIAVGLTCQHVSHAISVNHVKRAIAENLWSVCSECLKERRFYDGQLVLTSDIWLCLKCGFQGCGKNSESQHSLKHFKSSRTEPHCIIINLSTWIIWCYECDEKLSTHCNKKVLAQIVDFLQKHASKTQTSAFSRIMKLCEEKCETDEIQKGGKCRNLSVRGITNLGNTCFFNAVMQNLAQTYTLTDLMNEIKESSTKLKIFPSSDSQLDPLVVELSRPGPLTSALFLFLHSMKETEKGPLSPKVLFNQLCQKAPRFKDFQQQDSQELLHYLLDAVRTEETKRIQASILKAFNNPTTKTADDETRKKVKAYGKEGVKMNFIDRIFIGELTSTVMCEECANISTVKDPFIDISLPIIEERVSKPLLWGRMNKYRSLRETDHDRYSGNVTIENIHQPRAAKKHSSSKDKSQLIHDRKCIRKLSSGETVTYQKNENLEMNGDSLMFASLMNSESRLNESPTDDSEKEASHSESNVDADSEPSESESASKQTGLFRSSSGSGVQPDGPLYPLSAGKLLYTKETDSGDKEMAEAISELRLSSTVTGDQDFDRENQPLNISNNLCFLEGKHLRSYSPQNAFQTLSQSYITTSKECSIQSCLYQFTSMELLMGNNKLLCENCTKNKQKYQEETSFAEKKVEGVYTNARKQLLISAVPAVLILHLKRFHQAGLSLRKVNRHVDFPLMLDLAPFCSATCKNASVGDKVLYGLYGIVEHSGSMREGHYTAYVKVRTPSRKLSEHNTKKKNVPGLKAADNESAGQWVHVSDTYLQVVPESRALSAQAYLLFYERVL",
# UBP46_HUMAN
"MTVRNIASICNMGTNASALEKDIGPEQFPINEHYFGLVNFGNTCYCNSVLQALYFCRPFRENVLAYKAQQKKKENLLTCLADLFHSIATQKKKVGVIPPKKFISRLRKENDLFDNYMQQDAHEFLNYLLNTIADILQEEKKQEKQNGKLKNGNMNEPAENNKPELTWVHEIFQGTLTNETRCLNCETVSSKDEDFLDLSVDVEQNTSITHCLRDFSNTETLCSEQKYYCETCCSKQEAQKRMRVKKLPMILALHLKRFKYMEQLHRYTKLSYRVVFPLELRLFNTSSDAVNLDRMYDLVAVVVHCGSGPNRGHYITIVKSHGFWLLFDDDIVEKIDAQAIEEFYGLTSDISKNSESGYILFYQSRE",
# UBP47_HUMAN
"MVPGEENQLVPKEDVFWRCRQNIFDEMKKKFLQIENAAEEPRVLCIIQDTTNSKTVNERITLNLPASTPVRKLFEDVANKVGYINGTFDLVWGNGINTADMAPLDHTSDKSLLDANFEPGKKNFLHLTDKDGEQPQILLEDSSAGEDSVHDRFIGPLPREGSGGSTSDYVSQSYSYSSILNKSETGYVGLVNQAMTCYLNSLLQTLFMTPEFRNALYKWEFEESEEDPVTSIPYQLQRLFVLLQTSKKRAIETTDVTRSFGWDSSEAWQQHDVQELCRVMFDALEQKWKQTEQADLINELYQGKLKDYVRCLECGYEGWRIDTYLDIPLVIRPYGSSQAFASVEEALHAFIQPEILDGPNQYFCERCKKKCDARKGLRFLHFPYLLTLQLKRFDFDYTTMHRIKLNDRMTFPEELDMSTFIDVEDEKSPQTESCTDSGAENEGSCHSDQMSNDFSNDDGVDEGICLETNSGTEKISKSGLEKNSLIYELFSVMVHSGSAAGGHYYACIKSFSDEQWYSFNDQHVSRITQEDIKKTHGGSSGSRGYYSSAFASSTNAYMLIYRLKDPARNAKFLEVDEYPEHIKNLVQKERELEEQEKRQREIERNTCKIKLFCLHPTKQVMMENKLEVHKDKTLKEAVEMAYKMMDLEEVIPLDCCRLVKYDEFHDYLERSYEGEEDTPMGLLLGGVKSTYMFDLLLETRKPDQVFQSYKPGEVMVKVHVVDLKAESVAAPITVRAYLNQTVTEFKQLISKAIHLPAETMRIVLERCYNDLRLLSVSSKTLKAEGFFRSNKVFVESSETLDYQMAFADSHLWKLLDRHANTIRLFVLLPEQSPVSYSKRTAYQKAGGDSGNVDDDCERVKGPVGSLKSVEAILEESTEKLKSLSLQQQQDGDNGDSSKSTETSDFENIESPLNERDSSASVDNRELEQHIQTSDPENFQSEERSDSDVNNDRSTSSVDSDILSSSHSSDTLCNADNAQIPLANGLDSHSITSSRRTKANEGKKETWDTAEEDSGTDSEYDESGKSRGEMQYMYFKAEPYAADEGSGEGHKWLMVHVDKRITLAAFKQHLEPFVGVLSSHFKVFRVYASNQEFESVRLNETLSSFSDDNKITIRLGRALKKGEYRVKVYQLLVNEQEPCKFLLDAVFAKGMTVRQSKEELIPQLREQCGLELSIDRFRLRKKTWKNPGTVFLDYHIYEEDINISSNWEVFLEVLDGVEKMKSMSQLAVLSRRWKPSEMKLDPFQEVVLESSSVDELREKLSEISGIPLDDIEFAKGRGTFPCDISVLDIHQDLDWNPKVSTLNVWPLYICDDGAVIFYRDKTEELMELTDEQRNELMKKESSRLQKTGHRVTYSPRKEKALKIYLDGAPNKDLTQD",
# UBP48_HUMAN
"MAPRLQLEKAAWRWAETVRPEEVSQEHIETAYRIWLEPCIRGVCRRNCKGNPNCLVGIGEHIWLGEIDENSFHNIDDPNCERRKKNSFVGLTNLGATCYVNTFLQVWFLNLELRQALYLCPSTCSDYMLGDGIQEEKDYEPQTICEHLQYLFALLQNSNRRYIDPSGFVKALGLDTGQQQDAQEFSKLFMSLLEDTLSKQKNPDVRNIVQQQFCGEYAYVTVCNQCGRESKLLSKFYELELNIQGHKQLTDCISEFLKEEKLEGDNRYFCENCQSKQNATRKIRLLSLPCTLNLQLMRFVFDRQTGHKKKLNTYIGFSEILDMEPYVEHKGGSYVYELSAVLIHRGVSAYSGHYIAHVKDPQSGEWYKFNDEDIEKMEGKKLQLGIEEDLAEPSKSQTRKPKCGKGTHCSRNAYMLVYRLQTQEKPNTTVQVPAFLQELVDRDNSKFEEWCIEMAEMRKQSVDKGKAKHEEVKELYQRLPAGAEPYEFVSLEWLQKWLDESTPTKPIDNHACLCSHDKLHPDKISIMKRISEYAADIFYSRYGGGPRLTVKALCKECVVERCRILRLKNQLNEDYKTVNNLLKAAVKGSDGFWVGKSSLRSWRQLALEQLDEQDGDAEQSNGKMNGSTLNKDESKEERKEEEELNFNEDILCPHGELCISENERRLVSKEAWSKLQQYFPKAPEFPSYKECCSQCKILEREGEENEALHKMIANEQKTSLPNLFQDKNRPCLSNWPEDTDVLYIVSQFFVEEWRKFVRKPTRCSPVSSVGNSALLCPHGGLMFTFASMTKEDSKLIALIWPSEWQMIQKLFVVDHVIKITRIEVGDVNPSETQYISEPKLCPECREGLLCQQQRDLREYTQATIYVHKVVDNKKVMKDSAPELNVSSSETEEDKEEAKPDGEKDPDFNQSNGGTKRQKISHQNYIAYQKQVIRRSMRHRKVRGEKALLVSANQTLKELKIQIMHAFSVAPFDQNLSIDGKILSDDCATLGTLGVIPESVILLKADEPIADYAAMDDVMQVCMPEEGFKGTGLLGH",
# UBP49_HUMAN
"MDRCKHVGRLRLAQDHSILNPQKWCCLECATTESVWACLKCSHVACGRYIEDHALKHFEETGHPLAMEVRDLYVFCYLCKDYVLNDNPEGDLKLLRSSLLAVRGQKQDTPVRRGRTLRSMASGEDVVLPQRAPQGQPQMLTALWYRRQRLLARTLRLWFEKSSRGQAKLEQRRQEEALERKKEEARRRRREVKRRLLEELASTPPRKSARLLLHTPRDAGPAASRPAALPTSRRVPAATLKLRRQPAMAPGVTGLRNLGNTCYMNSILQVLSHLQKFRECFLNLDPSKTEHLFPKATNGKTQLSGKPTNSSATELSLRNDRAEACEREGFCWNGRASISRSLELIQNKEPSSKHISLCRELHTLFRVMWSGKWALVSPFAMLHSVWSLIPAFRGYDQQDAQEFLCELLHKVQQELESEGTTRRILIPFSQRKLTKQVLKVVNTIFHGQLLSQVTCISCNYKSNTIEPFWDLSLEFPERYHCIEKGFVPLNQTECLLTEMLAKFTETEALEGRIYACDQCNSKRRKSNPKPLVLSEARKQLMIYRLPQVLRLHLKRFRWSGRNHREKIGVHVVFDQVLTMEPYCCRDMLSSLDKETFAYDLSAVVMHHGKGFGSGHYTAYCYNTEGGFWVHCNDSKLNVCSVEEVCKTQAYILFYTQRTVQGNARISETHLQAQVQSSNNDEGRPQTFS",
# UBP5_HUMAN
"MAELSEEALLSVLPTIRVPKAGDRVHKDECAFSFDTPESEGGLYICMNTFLGFGKQYVERHFNKTGQRVYLHLRRTRRPKEEDPATGTGDPPRKKPTRLAIGVEGGFDLSEEKFELDEDVKIVILPDYLEIARDGLGGLPDIVRDRVTSAVEALLSADSASRKQEVQAWDGEVRQVSKHAFSLKQLDNPARIPPCGWKCSKCDMRENLWLNLTDGSILCGRRYFDGSGGNNHAVEHYRETGYPLAVKLGTITPDGADVYSYDEDDMVLDPSLAEHLSHFGIDMLKMQKTDKTMTELEIDMNQRIGEWELIQESGVPLKPLFGPGYTGIRNLGNSCYLNSVVQVLFSIPDFQRKYVDKLEKIFQNAPTDPTQDFSTQVAKLGHGLLSGEYSKPVPESGDGERVPEQKEVQDGIAPRMFKALIGKGHPEFSTNRQQDAQEFFLHLINMVERNCRSSENPNEVFRFLVEEKIKCLATEKVKYTQRVDYIMQLPVPMDAALNKEELLEYEEKKRQAEEEKMALPELVRAQVPFSSCLEAYGAPEQVDDFWSTALQAKSVAVKTTRFASFPDYLVIQIKKFTFGLDWVPKKLDVSIEMPEELDISQLRGTGLQPGEEELPDIAPPLVTPDEPKGSLGFYGNEDEDSFCSPHFSSPTSPMLDESVIIQLVEMGFPMDACRKAVYYTGNSGAEAAMNWVMSHMDDPDFANPLILPGSSGPGSTSAAADPPPEDCVTTIVSMGFSRDQALKALRATNNSLERAVDWIFSHIDDLDAEAAMDISEGRSAADSISESVPVGPKVRDGPGKYQLFAFISHMGTSTMCGHYVCHIKKEGRWVIYNDQKVCASEKPPKDLGYIYFYQRVAS",
# UBP50_HUMAN
"MTSQPSLPADDFDIYHVLAECTDYYDTLPVKEADGNQPHFQGVTGLWNLGNTCCVNAISQCLCSILPLVEYFLTGKYITALQKFLLPSDCSEVATAFAYLMTDMWLGDSDCVSPEIFWSALGNLYPAFTKKMQQDAQEFLICVLNELHEALKKYHYSRRRSYEKGSTQRCCRKWITTETSIITQLFEEQLNYSIVCLKCEKCTYKNEVFTVFSLPIPSKYECSLRDCLQCFFQQDALTWNNEIHCSFCETKQETAVRASISKAPKIIIFHLKRFDIQGTTKRKLRTDIHYPLTNLDLTPYICSIFRKYPKYNLCAVVNHFGDLDGGHYTAFCKNSVTQA",
# UBP51_HUMAN
"MAQVRETSLPSGSGVRWISGGGGGASPEEAVEKAGKMEEAAAGATKASSRREAEEMKLEPLQEREPAPEENLTWSSSGGDEKVLPSIPLRCHSSSSPVCPRRKPRPRPQPRARSRSQPGLSAPPPPPARPPPPPPPPPPPAPRPRAWRGSRRRSRPGSRPQTRRSCSGDLDGSGDPGGLGDWLLEVEFGQGPTGCSHVESFKVGKNWQKNLRLIYQRFVWSGTPETRKRKAKSCICHVCSTHMNRLHSCLSCVFFGCFTEKHIHKHAETKQHHLAVDLYHGVIYCFMCKDYVYDKDIEQIAKETKEKILRLLTSTSTDVSHQQFMTSGFEDKQSTCETKEQEPKLVKPKKKRRKKSVYTVGLRGLINLGNTCFMNCIVQALTHIPLLKDFFLSDKHKCIMTSPSLCLVCEMSSLFHAMYSGSRTPHIPYKLLHLIWIHAEHLAGYRQQDAHEFLIAILDVLHRHSKDDSGGQEANNPNCCNCIIDQIFTGGLQSDVTCQACHSVSTTIDPCWDISLDLPGSCATFDSQNPERADSTVSRDDHIPGIPSLTDCLQWFTRPEHLGSSAKIKCNSCQSYQESTKQLTMKKLPIVACFHLKRFEHVGKQRRKINTFISFPLELDMTPFLASTKESRMKEGQPPTDCVPNENKYSLFAVINHHGTLESGHYTSFIRQQKDQWFSCDDAIITKATIEDLLYSEGYLLFYHKQGLEKD",
# UBP53_HUMAN
"MAWVKFLRKPGGNLGKVYQPGSMLSLAPTKGLLNEPGQNSCFLNSAVQVLWQLDIFRRSLRVLTGHVCQGDACIFCALKTIFAQFQHSREKALPSDNIRHALAESFKDEQRFQLGLMDDAAECFENMLERIHFHIVPSRDADMCTSKSCITHQKFAMTLYEQCVCRSCGASSDPLPFTEFVRYISTTALCNEVERMLERHERFKPEMFAELLQAANTTDDYRKCPSNCGQKIKIRRVLMNCPEIVTIGLVWDSEHSDLTEAVVRNLATHLYLPGLFYRVTDENAKNSELNLVGMICYTSQHYCAFAFHTKSSKWVFFDDANVKEIGTRWKDVVSKCIRCHFQPLLLFYANPDGTAVSTEDALRQVISWSHYKSVAENMGCEKPVIHKSDNLKENGFGDQAKQRENQKFPTDNISSSNRSHSHTGVGKGPAKLSHIDQREKIKDISRECALKAIEQKNLLSSQRKDLEKGQRKDLGRHRDLVDEDLSHFQSGSPPAPNGFKQHGNPHLYHSQGKGSYKHDRVVPQSRASAQIISSSKSQILAPGEKITGKVKSDNGTGYDTDSSQDSRDRGNSCDSSSKSRNRGWKPMRETLNVDSIFSESEKRQHSPRHKPNISNKPKSSKDPSFSNWPKENPKQKGLMTIYEDEMKQEIGSRSSLESNGKGAEKNKGLVEGKVHGDNWQMQRTESGYESSDHISNGSTNLDSPVIDGNGTVMDISGVKETVCFSDQITTSNLNKERGDCTSLQSQHHLEGFRKELRNLEAGYKSHEFHPESHLQIKNHLIKRSHVHEDNGKLFPSSSLQIPKDHNAREHIHQSDEQKLEKPNECKFSEWLNIENSERTGLPFHVDNSASGKRVNSNEPSSLWSSHLRTVGLKPETAPLIQQQNIMDQCYFENSLSTECIIRSASRSDGCQMPKLFCQNLPPPLPPKKYAITSVPQSEKSESTPDVKLTEVFKATSHLPKHSLSTASEPSLEVSTHMNDERHKETFQVRECFGNTPNCPSSSSTNDFQANSGAIDAFCQPELDSISTCPNETVSLTTYFSVDSCMTDTYRLKYHQRPKLSFPESSGFCNNSLS",
# UBP54_HUMAN
"MSWKRNYFSGGRGSVQGMFAPRSSTSIAPSKGLSNEPGQNSCFLNSALQVLWHLDIFRRSFRQLTTHKCMGDSCIFCALKGIFNQFQCSSEKVLPSDTLRSALAKTFQDEQRFQLGIMDDAAECFENLLMRIHFHIADETKEDICTAQHCISHQKFAMTLFEQCVCTSCGATSDPLPFIQMVHYISTTSLCNQAICMLERREKPSPSMFGELLQNASTMGDLRNCPSNCGERIRIRRVLMNAPQIITIGLVWDSDHSDLAEDVIHSLGTCLKLGDLFFRVTDDRAKQSELYLVGMICYYGKHYSTFFFQTKIRKWMYFDDAHVKEIGPKWKDVVTKCIKGHYQPLLLLYADPQGTPVSTQDLPPQAEFQSYSRTCYDSEDSGREPSISSDTRTDSSTESYPYKHSHHESVVSHFSSDSQGTVIYNVENDSMSQSSRDTGHLTDSECNQKHTSKKGSLIERKRSSGRVRRKGDEPQASGYHSEGETLKEKQAPRNASKPSSSTNRLRDFKETVSNMIHNRPSLASQTNVGSHCRGRGGDQPDKKPPRTLPLHSRDWEIESTSSESKSSSSSKYRPTWRPKRESLNIDSIFSKDKRKHCGYTQLSPFSEDSAKEFIPDEPSKPPSYDIKFGGPSPQYKRWGPARPGSHLLEQHPRLIQRMESGYESSERNSSSPVSLDAALPESSNVYRDPSAKRSAGLVPSWRHIPKSHSSSILEVDSTASMGGWTKSQPFSGEEISSKSELDELQEEVARRAQEQELRRKREKELEAAKGFNPHPSRFMDLDELQNQGRSDGFERSLQEAESVFEESLHLEQKGDCAAALALCNEAISKLRLALHGASCSTHSRALVDKKLQISIRKARSLQDRMQQQQSPQQPSQPSACLPTQAGTLSQPTSEQPIPLQVLLSQEAQLESGMDTEFGASSFFHSPASCHESHSSLSPESSAPQHSSPSRSALKLLTSVEVDNIEPSAFHRQGLPKAPGWTEKNSHHSWEPLDAPEGKLQGSRCDNSSCSKLPPQEGRGIAQEQLFQEKKDPANPSPVMPGIATSERGDEHSLGCSPSNSSAQPSLPLYRTCHPIMPVASSFVLHCPDPVQKTNQCLQGQSLKTSLTLKVDRGSEETYRPEFPSTKGLVRSLAEQFQRMQGVSMRDSTGFKDRSLSGSLRKNSSPSDSKPPFSQGQEKGHWPWAKQQSSLEGGDRPLSWEESTEHSSLALNSGLPNGETSSGGQPRLAEPDIYQEKLSQVRDVRSKDLGSSTDLGTSLPLDSWVNITRFCDSQLKHGAPRPGMKSSPHDSHTCVTYPERNHILLHPHWNQDTEQETSELESLYQASLQASQAGCSGWGQQDTAWHPLSQTGSADGMGRRLHSAHDPGLSKTSTAEMEHGLHEARTVRTSQATPCRGLSRECGEDEQYSAENLRRISRSLSGTVVSEREEAPVSSHSFDSSNVRKPLETGHRCSSSSSLPVIHDPSVFLLGPQLYLPQPQFLSPDVLMPTMAGEPNRLPGTSRSVQQFLAMCDRGETSQGAKYTGRTLNYQSLPHRSRTDNSWAPWSETNQHIGTRFLTTPGCNPQLTYTATLPERSKGLQVPHTQSWSDLFHSPSHPPIVHPVYPPSSSLHVPLRSAWNSDPVPGSRTPGPRRVDMPPDDDWRQSSYASHSGHRRTVGEGFLFVLSDAPRREQIRARVLQHSQW",
# US6NL_HUMAN
"MNSDQDVALKLAQERAEIVAKYDRGREGAEIEPWEDADYLVYKVTDRFGFLHEEELPDHNVAVERQKHLEIERTTKWLKMLKGWEKYKNTEKFHRRIYKGIPLQLRGEVWALLLEIPKMKEETRDLYSKLKHRARGCSPDIRQIDLDVNRTFRDHIMFRDRYGVKQQSLFHVLAAYSIYNTEVGYCQGMSQITALLLMYMNEEDAFWALVKLFSGPKHAMHGFFVQGFPKLLRFQEHHEKILNKFLSKLKQHLDSQEIYTSFYTMKWFFQCFLDRTPFTLNLRIWDIYIFEGERVLTAMSYTILKLHKKHLMKLSMEELVEFFQETLAKDFFFEDDFVIEQLQISMTELKRAKLDLPEPGKEDEYPKKPLGQLPPELQSWGVHHLSNGQRSVGRPSPLASGRRESGAPHRRHEHSPHPQSRTGTPERAQPPRRKSVEEESKKLKDEADFQRKLPSGPQDSSRQYNHAAANQNSNATSNIRKEFVPKWNKPSDVSATERTAKYTMEGKGRAAHPALAVTVPGPAEVRVSNVRPKMKALDAEDGKRGSTASQYDNVPGPELDSGASVEEALERAYSQSPRHALYPPSPRKHAEPSSSPSKVSNKFTFKVQPPSHARYPSQLDGEARGLAHPPSYSNPPVYHGNSPKHFPTANSSFASPQFSPGTQLNPSRRPHGSTLSVSASPEKSYSRPSPLVLPSSRIEVLPVDTGAGGYSGNSGSPKNGKLIIPPVDYLPDNRTWSEVSYTYRPETQGQSWTRDASRGNLPKYSAFQLAPFQDHGLPAVSVDSPVRYKASPAAEDASPSGYPYSGPPPPAYHYRNRDGLSIQESVLL",
# UBP7_HUMAN
"MNHQQQQQQQKAGEQQLSEPEDMEMEAGDTDDPPRITQNPVINGNVALSDGHNTAEEDMEDDTSWRSEATFQFTVERFSRLSESVLSPPCFVRNLPWKIMVMPRFYPDRPHQKSVGFFLQCNAESDSTSWSCHAQAVLKIINYRDDEKSFSRRISHLFFHKENDWGFSNFMAWSEVTDPEKGFIDDDKVTFEVFVQADAPHGVAWDSKKHTGYVGLKNQGATCYMNSLLQTLFFTNQLRKAVYMMPTEGDDSSKSVPLALQRVFYELQHSDKPVGTKKLTKSFGWETLDSFMQHDVQELCRVLLDNVENKMKGTCVEGTIPKLFRGKMVSYIQCKEVDYRSDRREDYYDIQLSIKGKKNIFESFVDYVAVEQLDGDNKYDAGEHGLQEAEKGVKFLTLPPVLHLQLMRFMYDPQTDQNIKINDRFEFPEQLPLDEFLQKTDPKDPANYILHAVLVHSGDNHGGHYVVYLNPKGDGKWCKFDDDVVSRCTKEEAIEHNYGGHDDDLSVRHCTNAYMLVYIRESKLSEVLQAVTDHDIPQQLVERLQEEKRIEAQKRKERQEAHLYMQVQIVAEDQFCGHQGNDMYDEEKVKYTVFKVLKNSSLAEFVQSLSQTMGFPQDQIRLWPMQARSNGTKRPAMLDNEADGNKTMIELSDNENPWTIFLETVDPELAASGATLPKFDKDHDVMLFLKMYDPKTRSLNYCGHIYTPISCKIRDLLPVMCDRAGFIQDTSLILYEEVKPNLTERIQDYDVSLDKALDELMDGDIIVFQKDDPENDNSELPTAKEYFRDLYHRVDVIFCDKTIPNDPGFVVTLSNRMNYFQVAKTVAQRLNTDPMLLQFFKSQGYRDGPGNPLRHNYEGTLRDLLQFFKPRQPKKLYYQQLKMKITDFENRRSFKCIWLNSQFREEEITLYPDKHGCVRDLLEECKKAVELGEKASGKLRLLEIVSYKIIGVHQEDELLECLSPATSRTFRIEEIPLDQVDIDKENEMLVTVAHFHKEVFGTFGIPFLLRIHQGEHFREVMKRIQSLLDIQEKEFEKFKFAIVMMGRHQYINEDEYEVNLKDFEPQPGNMSHPRPWLGLDHFNKAPKRSRYTYLEKAIKIHN",
# UBP8_HUMAN
"MPAVASVPKELYLSSSLKDLNKKTEVKPEKISTKSYVHSALKIFKTAEECRLDRDEERAYVLYMKYVTVYNLIKKRPDFKQQQDYFHSILGPGNIKKAVEEAERLSESLKLRYEEAEVRKKLEEKDRQEEAQRLQQKRQETGREDGGTLAKGSLENVLDSKDKTQKSNGEKNEKCETKEKGAITAKELYTMMTDKNISLIIMDARRMQDYQDSCILHSLSVPEEAISPGVTASWIEAHLPDDSKDTWKKRGNVEYVVLLDWFSSAKDLQIGTTLRSLKDALFKWESKTVLRNEPLVLEGGYENWLLCYPQYTTNAKVTPPPRRQNEEVSISLDFTYPSLEESIPSKPAAQTPPASIEVDENIELISGQNERMGPLNISTPVEPVAASKSDVSPIIQPVPSIKNVPQIDRTKKPAVKLPEEHRIKSESTNHEQQSPQSGKVIPDRSTKPVVFSPTLMLTDEEKARIHAETALLMEKNKQEKELRERQQEEQKEKLRKEEQEQKAKKKQEAEENEITEKQQKAKEEMEKKESEQAKKEDKETSAKRGKEITGVKRQSKSEHETSDAKKSVEDRGKRCPTPEIQKKSTGDVPHTSVTGDSGSGKPFKIKGQPESGILRTGTFREDTDDTERNKAQREPLTRARSEEMGRIVPGLPSGWAKFLDPITGTFRYYHSPTNTVHMYPPEMAPSSAPPSTPPTHKAKPQIPAERDREPSKLKRSYSSPDITQAIQEEEKRKPTVTPTVNRENKPTCYPKAEISRLSASQIRNLNPVFGGSGPALTGLRNLGNTCYMNSILQCLCNAPHLADYFNRNCYQDDINRSNLLGHKGEVAEEFGIIMKALWTGQYRYISPKDFKITIGKINDQFAGYSQQDSQELLLFLMDGLHEDLNKADNRKRYKEENNDHLDDFKAAEHAWQKHKQLNESIIVALFQGQFKSTVQCLTCHKKSRTFEAFMYLSLPLASTSKCTLQDCLRLFSKEEKLTDNNRFYCSHCRARRDSLKKIEIWKLPPVLLVHLKRFSYDGRWKQKLQTSVDFPLENLDLSQYVIGPKNNLKKYNLFSVSNHYGGLDGGHYTAYCKNAARQRWFKFDDHEVSDISVSSVKSSAAYILFYTSLGPRVTDVAT",
# USP9X_HUMAN
"MTATTRGSPVGGNDNQGQAPDGQSQPPLQQNQTSSPDSSNENSPATPPDEQGQGDAPPQLEDEEPAFPHTDLAKLDDMINRPRWVVPVLPKGELEVLLEAAIDLSKKGLDVKSEACQRFFRDGLTISFTKILTDEAVSGWKFEIHRCIINNTHRLVELCVAKLSQDWFPLLELLAMALNPHCKFHIYNGTRPCESVSSSVQLPEDELFARSPDPRSPKGWLVDLLNKFGTLNGFQILHDRFINGSALNVQIIAALIKPFGQCYEFLTLHTVKKYFLPIIEMVPQFLENLTDEELKKEAKNEAKNDALSMIIKSLKNLASRVPGQEETVKNLEIFRLKMILRLLQISSFNGKMNALNEVNKVISSVSYYTHRHGNPEEEEWLTAERMAEWIQQNNILSIVLRDSLHQPQYVEKLEKILRFVIKEKALTLQDLDNIWAAQAGKHEAIVKNVHDLLAKLAWDFSPEQLDHLFDCFKASWTNASKKQREKLLELIRRLAEDDKDGVMAHKVLNLLWNLAHSDDVPVDIMDLALSAHIKILDYSCSQDRDTQKIQWIDRFIEELRTNDKWVIPALKQIREICSLFGEAPQNLSQTQRSPHVFYRHDLINQLQHNHALVTLVAENLATYMESMRLYARDHEDYDPQTVRLGSRYSHVQEVQERLNFLRFLLKDGQLWLCAPQAKQIWKCLAENAVYLCDREACFKWYSKLMGDEPDLDPDINKDFFESNVLQLDPSLLTENGMKCFERFFKAVNCREGKLVAKRRAYMMDDLELIGLDYLWRVVIQSNDDIASRAIDLLKEIYTNLGPRLQVNQVVIHEDFIQSCFDRLKASYDTLCVLDGDKDSVNCARQEAVRMVRVLTVLREYINECDSDYHEERTILPMSRAFRGKHLSFVVRFPNQGRQVDDLEVWSHTNDTIGSVRRCILNRIKANVAHTKIELFVGGELIDPADDRKLIGQLNLKDKSLITAKLTQISSNMPSSPDSSSDSSTGSPGNHGNHYSDGPNPEVESCLPGVIMSLHPRYISFLWQVADLGSSLNMPPLRDGARVLMKLMPPDSTTIEKLRAICLDHAKLGESSLSPSLDSLFFGPSASQVLYLTEVVYALLMPAGAPLADDSSDFQFHFLKSGGLPLVLSMLTRNNFLPNADMETRRGAYLNALKIAKLLLTAIGYGHVRAVAEACQPGVEGVNPMTQINQVTHDQAVVLQSALQSIPNPSSECMLRNVSVRLAQQISDEASRYMPDICVIRAIQKIIWASGCGSLQLVFSPNEEITKIYEKTNAGNEPDLEDEQVCCEALEVMTLCFALIPTALDALSKEKAWQTFIIDLLLHCHSKTVRQVAQEQFFLMCTRCCMGHRPLLFFITLLFTVLGSTARERAKHSGDYFTLLRHLLNYAYNSNINVPNAEVLLNNEIDWLKRIRDDVKRTGETGIEETILEGHLGVTKELLAFQTSEKKFHIGCEKGGANLIKELIDDFIFPASNVYLQYMRNGELPAEQAIPVCGSPPTINAGFELLVALAVGCVRNLKQIVDSLTEMYYIGTAITTCEALTEWEYLPPVGPRPPKGFVGLKNAGATCYMNSVIQQLYMIPSIRNGILAIEGTGSDVDDDMSGDEKQDNESNVDPRDDVFGYPQQFEDKPALSKTEDRKEYNIGVLRHLQVIFGHLAASRLQYYVPRGFWKQFRLWGEPVNLREQHDALEFFNSLVDSLDEALKALGHPAMLSKVLGGSFADQKICQGCPHRYECEESFTTLNVDIRNHQNLLDSLEQYVKGDLLEGANAYHCEKCNKKVDTVKRLLIKKLPPVLAIQLKRFDYDWERECAIKFNDYFEFPRELDMEPYTVAGVAKLEGDNVNPESQLIQQSEQSESETAGSTKYRLVGVLVHSGQASGGHYYSYIIQRNGGDGERNRWYKFDDGDVTECKMDDDEEMKNQCFGGEYMGEVFDHMMKRMSYRRQKRWWNAYILFYERMDTIDQDDELIRYISELAITTRPHQIIMPSAIERSVRKQNVQFMHNRMQYSMEYFQFMKKLLTCNGVYLNPPPGQDHLLPEAEEITMISIQLAARFLFTTGFHTKKVVRGSASDWYDALCILLRHSKNVRFWFAHNVLFNVSNRFSEYLLECPSAEVRGAFAKLIVFIAHFSLQDGPCPSPFASPGPSSQAYDNLSLSDHLLRAVLNLLRREVSEHGRHLQQYFNLFVMYANLGVAEKTQLLKLSVPATFMLVSLDEGPGPPIKYQYAELGKLYSVVSQLIRCCNVSSRMQSSINGNPPLPNPFGDPNLSQPIMPIQQNVADILFVRTSYVKKIIEDCSNSEETVKLLRFCCWENPQFSSTVLSELLWQVAYSYTYELRPYLDLLLQILLIEDSWQTHRIHNALKGIPDDRDGLFDTIQRSKNHYQKRAYQCIKCMVALFSNCPVAYQILQGNGDLKRKWTWAVEWLGDELERRPYTGNPQYTYNNWSPPVQSNETSNGYFLERSHSARMTLAKACELCPEEEPDDQDAPDEHESPPPEDAPLYPHSPGSQYQQNNHVHGQPYTGPAAHHMNNPQRTGQRAQENYEGSEEVSPPQTKDQ",
# USP9Y_HUMAN
"MTAITHGSPVGGNDSQGQVLDGQSQHLFQQNQTSSPDSSNENSVATPPPEEQGQGDAPPQHEDEEPAFPHTELANLDDMINRPRWVVPVLPKGELEVLLEAAIDLSVKGLDVKSEACQRFFRDGLTISFTKILMDEAVSGWKFEIHRCIINNTHRLVELCVAKLSQDWFPLLELLAMALNPHCKFHIYNGTRPCELISSNAQLPEDELFARSSDPRSPKGWLVDLINKFGTLNGFQILHDRFFNGSALNIQIIAALIKPFGQCYEFLSQHTLKKYFIPVIEIVPHLLENLTDEELKKEAKNEAKNDALSMIIKSLKNLASRISGQDETIKNLEIFRLKMILRLLQISSFNGKMNALNEINKVISSVSYYTHRHSNPEEEEWLTAERMAEWIQQNNILSIVLQDSLHQPQYVEKLEKILRFVIKEKALTLQDLDNIWAAQAGKHEAIVKNVHDLLAKLAWDFSPGQLDHLFDCFKASWTNASKKQREKLLELIRRLAEDDKDGVMAHKVLNLLWNLAQSDDVPVDIMDLALSAHIKILDYSCSQDRDAQKIQWIDHFIEELRTNDKWVIPALKQIREICSLFGEASQNLSQTQRSPHIFYRHDLINQLQQNHALVTLVAENLATYMNSIRLYAGDHEDYDPQTVRLGSRYSHVQEVQERLNFLRFLLKDGQLWLCAPQAKQIWKCLAENAVYLCDREACFKWYSKLMGDEPDLDPDINKDFFESNVLQLDPSLLTENGMKCFERFFKAVNCRERKLIAKRRSYMMDDLELIGLDYLWRVVIQSSDEIANRAIDLLKEIYTNLGPRLKANQVVIHEDFIQSCFDRLKASYDTLCVFDGDKNSINCARQEAIRMVRVLTVIKEYINECDSDYHKERMILPMSRAFRGKHLSLIVRFPNQGRQVDELDIWSHTNDTIGSVRRCIVNRIKANVAHKKIELFVGGELIDSEDDRKLIGQLNLKDKSLITAKLTQINFNMPSSPDSSSDSSTASPGNHRNHYNDGPNLEVESCLPGVIMSVHPRYISFLWQVADLGSNLNMPPLRDGARVLMKLMPPDRTAVEKLRAVCLDHAKLGEGKLSPPLDSLFFGPSASQVLYLTEVVYALLMPAGVPLTDGSSDFQVHFLKSGGLPLVLSMLIRNNFLPNTDMETRRGAYLNALKIAKLLLTAIGYGHVRAVAEACQPVVDGTDPITQINQVTHDQAVVLQSALQSIPNPSSECVLRNESILLAQEISNEASRYMPDICVIRAIQKIIWASACGALGLVFSPNEEITKIYQMTTNGSNKLEVEDEQVCCEALEVMTLCFALLPTALDALSKEKAWQTFIIDLLLHCPSKTVRQLAQEQFFLMCTRCCMGHRPLLFFITLLFTILGSTAREKGKYSGDYFTLLRHLLNYAYNGNINIPNAEVLLVSEIDWLKRIRDNVKNTGETGVEEPILEGHLGVTKELLAFQTSEKKYHFGCEKGGANLIKELIDDFIFPASKVYLQYLRSGELPAEQAIPVCSSPVTINAGFELLVALAIGCVRNLKQIVDCLTEMYYMGTAITTCEALTEWEYLPPVGPRPPKGFVGLKNAGATCYMNSVIQQLYMIPSIRNSILAIEGTGSDLHDDMFGDEKQDSESNVDPRDDVFGYPHQFEDKPALSKTEDRKEYNIGVLRHLQVIFGHLAASQLQYYVPRGFWKQFRLWGEPVNLREQHDALEFFNSLVDSLDEALKALGHPAILSKVLGGSFADQKICQGCPHRYECEESFTTLNVDIRNHQNLLDSLEQYIKGDLLEGANAYHCEKCDKKVDTVKRLLIKKLPRVLAIQLKRFDYDWERECAIKFNDYFEFPRELDMGPYTVAGVANLERDNVNSENELIEQKEQSDNETAGGTKYRLVGVLVHSGQASGGHYYSYIIQRNGKDDQTDHWYKFDDGDVTECKMDDDEEMKNQCFGGEYMGEVFDHMMKRMSYRRQKRWWNAYILFYEQMDMIDEDDEMIRYISELTIARPHQIIMSPAIERSVRKQNVKFMHNRLQYSLEYFQFVKKLLTCNGVYLNPAPGQDYLLPEAEEITMISIQLAARFLFTTGFHTKKIVRGPASDWYDALCVLLRHSKNVRFWFTHNVLFNVSNRFSEYLLECPSAEVRGAFAKLIVFIAHFSLQDGSCPSPFASPGPSSQACDNLSLSDHLLRATLNLLRREVSEHGHHLQQYFNLFVMYANLGVAEKTQLLKLNVPATFMLVSLDEGPGPPIKYQYAELGKLYSVVSQLIRCCNVSSTMQSSINGNPPLPNPFGDLNLSQPIMPIQQNVLDILFVRTSYVKKIIEDCSNSEDTIKLLRFCSWENPQFSSTVLSELLWQVAYSYTYELRPYLDLLFQILLIEDSWQTHRIHNALKGIPDDRDGLFDTIQRSKNHYQKRAYQCIKCMVALFSSCPVAYQILQGNGDLKRKWTWAVEWLGDELERRPYTGNPQYSYNNWSPPVQSNETANGYFLERSHSARMTLAKACELCPEEEPDDQDAPDEHEPSPSEDAPLYPHSPASQYQQNNHVHGQPYTGPAAHHLNNPQKTGQRTQENYEGNEEVSSPQMKDQ",
# USPL1_HUMAN
"MMDSPKIGNGLPVIGPGTDIGISSLHMVGYLGKNFDSAKVPSDEYCPACREKGKLKALKTYRISFQESIFLCEDLQCIYPLGSKSLNNLISPDLEECHTPHKPQKRKSLESSYKDSLLLANSKKTRNYIAIDGGKVLNSKHNGEVYDETSSNLPDSSGQQNPIRTADSLERNEILEADTVDMATTKDPATVDVSGTGRPSPQNEGCTSKLEMPLESKCTSFPQALCVQWKNAYALCWLDCILSALVHSEELKNTVTGLCSKEESIFWRLLTKYNQANTLLYTSQLSGVKDGDCKKLTSEIFAEIETCLNEVRDEIFISLQPQLRCTLGDMESPVFAFPLLLKLETHIEKLFLYSFSWDFECSQCGHQYQNRHMKSLVTFTNVIPEWHPLNAAHFGPCNNCNSKSQIRKMVLEKVSPIFMLHFVEGLPQNDLQHYAFHFEGCLYQITSVIQYRANNHFITWILDADGSWLECDDLKGPCSERHKKFEVPASEIHIVIWERKISQVTDKEAACLPLKKTNDQHALSNEKPVSLTSCSVGDAASAETASVTHPKDISVAPRTLSQDTAVTHGDHLLSGPKGLVDNILPLTLEETIQKTASVSQLNSEAFLLENKPVAENTGILKTNTLLSQESLMASSVSAPCNEKLIQDQFVDISFPSQVVNTNMQSVQLNTEDTVNTKSVNNTDATGLIQGVKSVEIEKDAQLKQFLTPKTEQLKPERVTSQVSNLKKKETTADSQTTTSKSLQNQSLKENQKKPFVGSWVKGLISRGASFMPLCVSAHNRNTITDLQPSVKGVNNFGGFKTKGINQKASHVSKKARKSASKPPPISKPPAGPPSSNGTAAHPHAHAASEVLEKSGSTSCGAQLNHSSYGNGISSANHEDLVEGQIHKLRLKLRKKLKAEKKKLAALMSSPQSRTVRSENLEQVPQDGSPNDCESIEDLLNELPYPIDIASESACTTVPGVSLYSSQTHEEILAELLSPTPVSTELSENGEGDFRYLGMGDSHIPPPVPSEFNDVSQNTHLRQDHNYCSPTKKNPCEVQPDSLTNNACVRTLNLESPMKTDIFDEFFSSSALNALANDTLDLPHFDEYLFENY",
# VCIP1_HUMAN
"MSQPPPPPPPLPPPPPPPEAPQTPSSLASAAASGGLLKRRDRRILSGSCPDPKCQARLFFPASGSVSIECTECGQRHEQQQLLGVEEVTDPDVVLHNLLRNALLGVTGAPKKNTELVKVMGLSNYHCKLLSPILARYGMDKQTGRAKLLRDMNQGELFDCALLGDRAFLIEPEHVNTVGYGKDRSGSLLYLHDTLEDIKRANKSQECLIPVHVDGDGHCLVHAVSRALVGRELFWHALRENLKQHFQQHLARYQALFHDFIDAAEWEDIINECDPLFVPPEGVPLGLRNIHIFGLANVLHRPIILLDSLSGMRSSGDYSATFLPGLIPAEKCTGKDGHLNKPICIAWSSSGRNHYIPLVGIKGAALPKLPMNLLPKAWGVPQDLIKKYIKLEEDGGCVIGGDRSLQDKYLLRLVAAMEEVFMDKHGIHPSLVADVHQYFYRRTGVIGVQPEEVTAAAKKAVMDNRLHKCLLCGALSELHVPPEWLAPGGKLYNLAKSTHGQLRTDKNYSFPLNNLVCSYDSVKDVLVPDYGMSNLTACNWCHGTSVRKVRGDGSIVYLDGDRTNSRSTGGKCGCGFKHFWDGKEYDNLPEAFPITLEWGGRVVRETVYWFQYESDSSLNSNVYDVAMKLVTKHFPGEFGSEILVQKVVHTILHQTAKKNPDDYTPVNIDGAHAQRVGDVQGQESESQLPTKIILTGQKTKTLHKEELNMSKTERTIQQNITEQASVMQKRKTEKLKQEQKGQPRTVSPSTIRDGPSSAPATPTKAPYSPTTSKEKKIRITTNDGRQSMVTLKSSTTFFELQESIAREFNIPPYLQCIRYGFPPKELMPPQAGMEKEPVPLQHGDRITIEILKSKAEGGQSAAAHSAHTVKQEDIAVTGKLSSKELQEQAEKEMYSLCLLATLMGEDVWSYAKGLPHMFQQGGVFYSIMKKTMGMADGKHCTFPHLPGKTFVYNASEDRLELCVDAAGHFPIGPDVEDLVKEAVSQVRAEATTRSRESSPSHGLLKLGSGGVVKKKSEQLHNVTAFQGKGHSLGTASGNPHLDPRARETSVVRKHNTGTDFSNSSTKTEPSVFTASSSNSELIRIAPGVVTMRDGRQLDPDLVEAQRKKLQEMVSSIQASMDRHLRDQSTEQSPSDLPQRKTEVVSSSAKSGSLQTGLPESFPLTGGTENLNTETTDGCVADALGAAFATRSKAQRGNSVEELEEMDSQDAEMTNTTEPMDHS",
# OTU1_HUMAN
"MFGPAKGRHFGVHPAPGFPGGVSQQAAGTKAGPAGAWPVGSRTDTMWRLRCKAKDGTHVLQGLSSRTRVRELQGQIAAITGIAPGGQRILVGYPPECLDLSNGDTILEDLPIQSGDMLIIEEDQTRPRSSPAFTKRGASSYVRETLPVLTRTVVPADNSCLFTSVYYVVEGGVLNPACAPEMRRLIAQIVASDPDFYSEAILGKTNQEYCDWIKRDDTWGGAIEISILSKFYQCEICVVDTQTVRIDRFGEDAGYTKRVLLIYDGIHYDPLQRNFPDPDTPPLTIFSSNDDIVLVQALELADEARRRRQFTDVNRFTLRCMVCQKGLTGQAEAREHAKETGHTNFGEV",
# ZRAN1_HUMAN
"MSERGIKWACEYCTYENWPSAIKCTMCRAQRPSGTIITEDPFKSGSSDVGRDWDPSSTEGGSSPLICPDSSARPRVKSSYSMENANKWSCHMCTYLNWPRAIRCTQCLSQRRTRSPTESPQSSGSGSRPVAFSVDPCEEYNDRNKLNTRTQHWTCSVCTYENWAKAKRCVVCDHPRPNNIEAIELAETEEASSIINEQDRARWRGSCSSGNSQRRSPPATKRDSEVKMDFQRIELAGAVGSKEELEVDFKKLKQIKNRMKKTDWLFLNACVGVVEGDLAAIEAYKSSGGDIARQLTADEVRLLNRPSAFDVGYTLVHLAIRFQRQDMLAILLTEVSQQAAKCIPAMVCPELTEQIRREIAASLHQRKGDFACYFLTDLVTFTLPADIEDLPPTVQEKLFDEVLDRDVQKELEEESPIINWSLELATRLDSRLYALWNRTAGDCLLDSVLQATWGIYDKDSVLRKALHDSLHDCSHWFYTRWKDWESWYSQSFGLHFSLREEQWQEDWAFILSLASQPGASLEQTHIFVLAHILRRPIIVYGVKYYKSFRGETLGYTRFQGVYLPLLWEQSFCWKSPIALGYTRGHFSALVAMENDGYGNRGAGANLNTDDDVTITFLPLVDSERKLLHVHFLSAQELGNEEQQEKLLREWLDCCVTEGGVLVAMQKSSRRRNHPLVTQMVEKWLDRYRQIRPCTSLSDGEEDEDDEDE"
]


# Add protein names corresponding to the sequences
protein_names = [
"AFF4_HUMAN",
"AMFR_HUMAN",
"APC11_HUMAN",
"AKIB1_HUMAN",
"AREL1_HUMAN",
"ARI1_HUMAN",
"ARI2_HUMAN",
"BARD1_HUMAN",
"BFAR_HUMAN",
"BIRC2_HUMAN",
"BIRC3_HUMAN",
"BIRC7_HUMAN",
"BIRC8_HUMAN",
"BMI1_HUMAN",
"BRAP_HUMAN",
"BRCA1_HUMAN",
"CBL_HUMAN",
"CBLB_HUMAN",
"CBLC_HUMAN",
"HAKAI_HUMAN",
"IHO1_HUMAN",
"CIP1_HUMAN",
"CGRF1_HUMAN",
"CHFR_HUMAN",
"CNOT4_HUMAN",
"CUL9_HUMAN",
"TM276_HUMAN",
"ZTRF1_HUMAN",
"DCST1_HUMAN",
"DTX1_HUMAN",
"DTX2_HUMAN",
"DTX3_HUMAN",
"DTX3L_HUMAN",
"DTX4_HUMAN",
"DZIP3_HUMAN",
"E4F1_HUMAN",
"FANCL_HUMAN",
"G2E3_HUMAN",
"HACE1_HUMAN",
"HECD1_HUMAN",
"HECD2_HUMAN",
"HECD3_HUMAN",
"HECD4_HUMAN",
"HECW1_HUMAN",
"HECW2_HUMAN",
"HERC1_HUMAN",
"HERC2_HUMAN",
"HERC3_HUMAN",
"HERC4_HUMAN",
"HERC5_HUMAN",
"HERC6_HUMAN",
"HLTF_HUMAN",
"HUWE1_HUMAN",
"I2BP1_HUMAN",
"I2BP2_HUMAN",
"I2BPL_HUMAN",
"ITCH_HUMAN",
"KCMF1_HUMAN",
"KMT2C_HUMAN",
"KMT2D_HUMAN",
"LNX1_HUMAN",
"LNX2_HUMAN",
"LONF1_HUMAN",
"LONF2_HUMAN",
"LONF3_HUMAN",
"LRSM1_HUMAN",
"LTN1_HUMAN",
"MAEA_HUMAN",
"M3K1_HUMAN",
"MARH1_HUMAN",
"MARHA_HUMAN",
"MARHB_HUMAN",
"MARH2_HUMAN",
"MARH3_HUMAN",
"MARH4_HUMAN",
"MARH5_HUMAN",
"MARH6_HUMAN",
"MARH7_HUMAN",
"MARH8_HUMAN",
"MARH9_HUMAN",
"MDM2_HUMAN",
"MDM4_HUMAN",
"MECOM_HUMAN",
"MEX3A_HUMAN",
"MEX3B_HUMAN",
"MEX3C_HUMAN",
"MEX3D_HUMAN",
"MGRN1_HUMAN",
"MIB1_HUMAN",
"MIB2_HUMAN",
"TRI18_HUMAN",
"TRIM1_HUMAN",
"MKRN1_HUMAN",
"MKRN2_HUMAN",
"MKRN3_HUMAN",
"MKRN4_HUMAN",
"MAT1_HUMAN",
"COAC_HUMAN",
"MUL1_HUMAN",
"MYCB2_HUMAN",
"MYLIP_HUMAN",
"NEDD4_HUMAN",
"NED4L_HUMAN",
"NEUL1_HUMAN",
"NEU1B_HUMAN",
"NEUL3_HUMAN",
"NFX1_HUMAN",
"NFXL1_HUMAN",
"NHLC1_HUMAN",
"NOSIP_HUMAN",
"NSE1_HUMAN",
"PRKN_HUMAN",
"PCGF1_HUMAN",
"PCGF2_HUMAN",
"PCGF3_HUMAN",
"PCGF5_HUMAN",
"PCGF6_HUMAN",
"PZRN3_HUMAN",
"PZRN4_HUMAN",
"PELI1_HUMAN",
"PELI2_HUMAN",
"PELI3_HUMAN",
"PEX10_HUMAN",
"PEX12_HUMAN",
"PEX2_HUMAN",
"PHF7_HUMAN",
"PHRF1_HUMAN",
"PJA1_HUMAN",
"PJA2_HUMAN",
"PLAG1_HUMAN",
"PLAL1_HUMAN",
"PML_HUMAN",
"PPIL2_HUMAN",
"PRP19_HUMAN",
"RAD18_HUMAN",
"RAG1_HUMAN",
"RAPSN_HUMAN",
"RBBP6_HUMAN",
"HOIL1_HUMAN",
"RBX1_HUMAN",
"RC3H1_HUMAN",
"RC3H2_HUMAN",
"ZN363_HUMAN",
"RFFL_HUMAN",
"RFPL1_HUMAN",
"RFPL2_HUMAN",
"RFPL3_HUMAN",
"RFPLA_HUMAN",
"RFAL1_HUMAN",
"RFPLB_HUMAN",
"COP1_HUMAN",
"RFWD3_HUMAN",
"RING1_HUMAN",
"RLF_HUMAN",
"RNF12_HUMAN",
"RMD5A_HUMAN",
"RMD5B_HUMAN",
"RNF10_HUMAN",
"RN103_HUMAN",
"RNF11_HUMAN",
"RN111_HUMAN",
"RN112_HUMAN",
"R113A_HUMAN",
"R113B_HUMAN",
"RN114_HUMAN",
"RN115_HUMAN",
"RN121_HUMAN",
"RN122_HUMAN",
"RN123_HUMAN",
"RN125_HUMAN",
"RN126_HUMAN",
"RN128_HUMAN",
"MENB_BACSU",
"GOLI_HUMAN",
"RN133_HUMAN",
"RN135_HUMAN",
"RN138_HUMAN",
"RN139_HUMAN",
"RNF14_HUMAN",
"RN141_HUMAN",
"R144A_HUMAN",
"R144B_HUMAN",
"RN145_HUMAN",
"RN146_HUMAN",
"RN148_HUMAN",
"RN149_HUMAN",
"RN150_HUMAN",
"RN151_HUMAN",
"RN152_HUMAN",
"RN157_HUMAN",
"ARK2C_HUMAN",
"RN166_HUMAN",
"RN167_HUMAN",
"RN168_HUMAN",
"RN169_HUMAN",
"RNF17_HUMAN",
"RN170_HUMAN",
"RN175_HUMAN",
"RN180_HUMAN",
"RN181_HUMAN",
"RN182_HUMAN",
"RN183_HUMAN",
"RN185_HUMAN",
"RN186_HUMAN",
"RN187_HUMAN",
"RN19A_HUMAN",
"RN19B_HUMAN",
"RING2_HUMAN",
"BRE1A_HUMAN",
"RN207_HUMAN",
"RN208_HUMAN",
"RN212_HUMAN",
"R212B_HUMAN",
"RN213_HUMAN",
"RN214_HUMAN",
"RN215_HUMAN",
"RN216_HUMAN",
"RN217_HUMAN",
"OBI1_HUMAN",
"RN220_HUMAN",
"RN222_HUMAN",
"RN223_HUMAN",
"RN224_HUMAN",
"RN225_HUMAN",
"RNF24_HUMAN",
"RNF25_HUMAN",
"RNF26_HUMAN",
"RNF31_HUMAN",
"RNF32_HUMAN",
"RNF34_HUMAN",
"RNF38_HUMAN",
"RNF39_HUMAN",
"RNF4_HUMAN",
"BRE1B_HUMAN",
"RNF41_HUMAN",
"RNF43_HUMAN",
"RNF44_HUMAN",
"RNF5_HUMAN",
"RNF6_HUMAN",
"RBX2_HUMAN",
"RNF8_HUMAN",
"RNFT1_HUMAN",
"RNFT2_HUMAN",
"RSPRY_HUMAN",
"SCAFB_HUMAN",
"SH3R1_HUMAN",
"SH3R2_HUMAN",
"SH3R3_HUMAN",
"SHPRH_HUMAN",
"SIAH1_HUMAN",
"SIAH2_HUMAN",
"SIAH3_HUMAN",
"SMUF1_HUMAN",
"SMUF2_HUMAN",
"CHIP_HUMAN",
"SYVN1_HUMAN",
"TM129_HUMAN",
"TOPRS_HUMAN",
"TRAF2_HUMAN",
"TRAF3_HUMAN",
"TRAF4_HUMAN",
"TRAF5_HUMAN",
"TRAF6_HUMAN",
"TRAF7_HUMAN",
"TRAIP_HUMAN",
"TRI10_HUMAN",
"TRI11_HUMAN",
"TRI13_HUMAN",
"TRI15_HUMAN",
"TRI17_HUMAN",
"TRIM2_HUMAN",
"RO52_HUMAN",
"TRI22_HUMAN",
"TRI23_HUMAN",
"TIF1A_HUMAN",
"TRI25_HUMAN",
"TRI26_HUMAN",
"TRI27_HUMAN",
"TRIM3_HUMAN",
"TRI31_HUMAN",
"TRI32_HUMAN",
"TRI33_HUMAN",
"TRI34_HUMAN",
"TRI35_HUMAN",
"TRI36_HUMAN",
"TRI37_HUMAN",
"TRI38_HUMAN",
"TRI39_HUMAN",
"TRIM4_HUMAN",
"TRI40_HUMAN",
"TRI41_HUMAN",
"TRI42_HUMAN",
"TRI43_HUMAN",
"TR43B_HUMAN",
"TRI45_HUMAN",
"TRI46_HUMAN",
"TRI47_HUMAN",
"TRI48_HUMAN",
"TRI49_HUMAN",
"TR49B_HUMAN",
"TR49C_HUMAN",
"TR49D_HUMAN",
"TRIM5_HUMAN",
"TRI50_HUMAN",
"TRI51_HUMAN",
"TRI52_HUMAN",
"TRI54_HUMAN",
"TRI55_HUMAN",
"TRI56_HUMAN",
"TRI58_HUMAN",
"TRI59_HUMAN",
"TRIM6_HUMAN",
"TRI60_HUMAN",
"TRI61_HUMAN",
"TRI62_HUMAN",
"TRI63_HUMAN",
"TRI64_HUMAN",
"TR64B_HUMAN",
"TR64C_HUMAN",
"TRI65_HUMAN",
"TRI67_HUMAN",
"TRI68_HUMAN",
"TRI69_HUMAN",
"TRIM7_HUMAN",
"LIN41_HUMAN",
"TRI72_HUMAN",
"TRI73_HUMAN",
"TRI74_HUMAN",
"TRI75_HUMAN",
"TRI77_HUMAN",
"TRIM8_HUMAN",
"TRIM9_HUMAN",
"TRIML_HUMAN",
"TRIMM_HUMAN",
"TRIPC_HUMAN",
"TTC3_HUMAN",
"UBE3A_HUMAN",
"UBE3B_HUMAN",
"UBE3C_HUMAN",
"UBE3D_HUMAN",
"UBE4A_HUMAN",
"UBE4B_HUMAN",
"RNF37_HUMAN",
"UBR1_HUMAN",
"UBR2_HUMAN",
"UBR3_HUMAN",
"UBR4_HUMAN",
"UBR5_HUMAN",
"UBR7_HUMAN",
"UHRF1_HUMAN",
"UHRF2_HUMAN",
"UNK_HUMAN",
"UNKL_HUMAN",
"VPS11_HUMAN",
"VPS18_HUMAN",
"VPS41_HUMAN",
"VPS8_HUMAN",
"WDR59_HUMAN",
"WSDU1_HUMAN",
"WWP1_HUMAN",
"WWP2_HUMAN",
"XIAP_HUMAN",
"ZBT12_HUMAN",
"ZFP91_HUMAN",
"ZFPL1_HUMAN",
"Z280A_HUMAN",
"ZN341_HUMAN",
"ZN511_HUMAN",
"ZN521_HUMAN",
"ZN598_HUMAN",
"CBLL2_HUMAN",
"ZNRF1_HUMAN",
"ZNRF2_HUMAN",
"ZNRF3_HUMAN",
"ZNRF4_HUMAN",
"ZSWM2_HUMAN",
"ZXDC_HUMAN",
"ATX3_HUMAN",
"ATX3L_HUMAN",
"BAP1_HUMAN",
"BRCC3_HUMAN",
"CSN5_HUMAN",
"CSN6_HUMAN",
"CYLD_HUMAN",
"DESI1_HUMAN",
"DESI2_HUMAN",
"EIF3F_HUMAN",
"EIF3H_HUMAN",
"JOS1_HUMAN",
"JOS2_HUMAN",
"MPND_HUMAN",
"MYSM1_HUMAN",
"OTUB1_HUMAN",
"OTUB2_HUMAN",
"OTUD1_HUMAN",
"OTUD3_HUMAN",
"OTUD4_HUMAN",
"OTUD5_HUMAN",
"OTU6A_HUMAN",
"OTU6B_HUMAN",
"OTU7A_HUMAN",
"OTU7B_HUMAN",
"OTUL_HUMAN",
"PAN2_HUMAN",
"PAR11_HUMAN",
"PRP8_HUMAN",
"PSDE_HUMAN",
"PSMD7_HUMAN",
"SENP1_HUMAN",
"SENP2_HUMAN",
"SENP3_HUMAN",
"SENP5_HUMAN",
"SENP6_HUMAN",
"SENP7_HUMAN",
"SENP8_HUMAN",
"STABP_HUMAN",
"STALP_HUMAN",
"UCHL1_HUMAN",
"UCHL3_HUMAN",
"UCHL4_MOUSE",
"UCHL5_HUMAN",
"UBP1_HUMAN",
"UBP10_HUMAN",
"UBP11_HUMAN",
"UBP12_HUMAN",
"UBP13_HUMAN",
"UBP14_HUMAN",
"UBP15_HUMAN",
"UBP16_HUMAN",
"U17LA_HUMAN",
"U17LB_HUMAN",
"UL17C_HUMAN",
"U17LD_HUMAN",
"U17LF_HUMAN",
"U17LH_HUMAN",
"U17LI_HUMAN",
"U17LJ_HUMAN",
"U17LK_HUMAN",
"U17LL_HUMAN",
"U17LM_HUMAN",
"U17L3_HUMAN",
"U17L4_HUMAN",
"U17L5_HUMAN",
"U17L7_HUMAN",
"U17L8_HUMAN",
"U17PA_MOUSE",
"U17PB_MOUSE",
"U17LC_MOUSE",
"U17PD_MOUSE",
"U17PE_MOUSE",
"UBP18_HUMAN",
"UBP19_HUMAN",
"UBP2_HUMAN",
"UBP20_HUMAN",
"UBP21_HUMAN",
"UBP22_HUMAN",
"UBP24_HUMAN",
"UBP25_HUMAN",
"UBP26_HUMAN",
"UBP27_HUMAN",
"UBP28_HUMAN",
"UBP29_HUMAN",
"UBP3_HUMAN",
"UBP30_HUMAN",
"UBP31_HUMAN",
"UBP32_HUMAN",
"UBP33_HUMAN",
"UBP34_HUMAN",
"UBP35_HUMAN",
"UBP36_HUMAN",
"UBP37_HUMAN",
"UBP38_HUMAN",
"UBP39_HUMAN",
"UBP4_HUMAN",
"UBP40_HUMAN",
"UBP42_HUMAN",
"UBP43_HUMAN",
"UBP44_HUMAN",
"UBP45_HUMAN",
"UBP46_HUMAN",
"UBP47_HUMAN",
"UBP48_HUMAN",
"UBP49_HUMAN",
"UBP5_HUMAN",
"UBP50_HUMAN",
"UBP51_HUMAN",
"UBP53_HUMAN",
"UBP54_HUMAN",
"US6NL_HUMAN",
"UBP7_HUMAN",
"UBP8_HUMAN",
"USP9X_HUMAN",
"USP9Y_HUMAN",
"USPL1_HUMAN",
"VCIP1_HUMAN",
"OTU1_HUMAN",
"ZRAN1_HUMAN"
]

In [68]:
#Kinase
sequences = [
# ABL1_HUMAN
"MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSSTAFIPLISTRVSLRKTRQPPERIASGAITKGVVLDSTEALCLAISRNSEQMASHSAVLEAGKNLYTFCVSYVDSIQQMRNKFAFREAINKLENNLRELQICPATAGSGPAATQDFSKLLSSVKEISDIVQR",
# ACK1_HUMAN
"MQPEEGTGWLLELLSEVQLQQYFLRLRDDLNVTRLSHFEYVKNEDLEKIGMGRPGQRRLWEAVKRRKALCKRKSWMSKVFSGKRLEAEFPPHHSQSTFRKTSPAPGGPAGEGPLQSLTCLIGEKDLRLLEKLGDGSFGVVRRGEWDAPSGKTVSVAVKCLKPDVLSQPEAMDDFIREVNAMHSLDHRNLIRLYGVVLTPPMKMVTELAPLGSLLDRLRKHQGHFLLGTLSRYAVQVAEGMGYLESKRFIHRDLAARNLLLATRDLVKIGDFGLMRALPQNDDHYVMQEHRKVPFAWCAPESLKTRTFSHASDTWMFGVTLWEMFTYGQEPWIGLNGSQILHKIDKEGERLPRPEDCPQDIYNVMVQCWAHKPEDRPTFVALRDFLLEAQPTDMRALQDFEEPDKLHIQMNDVITVIEGRAENYWWRGQNTRTLCVGPFPRNVVTSVAGLSAQDISQPLQNSFIHTGHGDSDPRHCWGFPDRIDELYLGNPMDPPDLLSVELSTSRPPQHLGGVKKPTYDPVSEDQDPLSSDFKRLGLRKPGLPRGLWLAKPSARVPGTKASRGSGAEVTLIDFGEEPVVPALRPCAPSLAQLAMDACSLLDETPPQSPTRALPRPLHPTPVVDWDARPLPPPPAYDDVAQDEDDFEICSINSTLVGAGVPAGPSQGQTNYAFVPEQARPPPPLEDNLFLPPQGGGKPPSSAQTAEIFQALQQECMRQLQAPAGSPAPSPSPGGDDKPQVPPRVPIPPRPTRPHVQLSPAPPGEEETSQWPGPASPPRVPPREPLSPQGSRTPSPLVPPGSSPLPPRLSSSPGKTMPTTQSFASDPKYATPQVIQAPGPRAGPCILPIVRDGKKVSSTHYYLLPERPSYLERYQRFLREAQSPEEPTPLPVPLLLPPPSTPAPAAPTATVRPMPQAALDPKANFSTNNSNPGARPPPPRATARLPQRGCPGDGPEAGRPADKIQMAMVHGVTTEECQAALQCHGWSVQRAAQYLKVEQLFGLGLRPRGECHKVLEMFDWNLEQAGCHLLGSWGPAHHKR",
# AVR2A_HUMAN
"MGAAAKLAFAVFLISCSSGAILGRSETQECLFFNANWEKDRTNQTGVEPCYGDKDKRRHCFATWKNISGSIEIVKQGCWLDDINCYDRTDCVEKKDSPEVYFCCCEGNMCNEKFSYFPEMEVTQPTSNPVTPKPPYYNILLYSLVPLMLIAGIVICAFWVYRHHKMAYPPVLVPTQDPGPPPPSPLLGLKPLQLLEVKARGRFGCVWKAQLLNEYVAVKIFPIQDKQSWQNEYEVYSLPGMKHENILQFIGAEKRGTSVDVDLWLITAFHEKGSLSDFLKANVVSWNELCHIAETMARGLAYLHEDIPGLKDGHKPAISHRDIKSKNVLLKNNLTACIADFGLALKFEAGKSAGDTHGQVGTRRYMAPEVLEGAINFQRDAFLRIDMYAMGLVLWELASRCTAADGPVDEYMLPFEEEIGQHPSLEDMQEVVVHKKKRPVLRDYWQKHAGMAMLCETIEECWDHDAEARLSAGCVGERITQMQRLTNIITTEDIVTVVTMVTNVDFPPKESSL",
# AVR2B_HUMAN
"MTAPWVALALLWGSLCAGSGRGEAETRECIYYNANWELERTNQSGLERCEGEQDKRLHCYASWRNSSGTIELVKKGCWLDDFNCYDRQECVATEENPQVYFCCCEGNFCNERFTHLPEAGGPEVTYEPPPTAPTLLTVLAYSLLPIGGLSLIVLLAFWMYRHRKPPYGHVDIHEDPGPPPPSPLVGLKPLQLLEIKARGRFGCVWKAQLMNDFVAVKIFPLQDKQSWQSEREIFSTPGMKHENLLQFIAAEKRGSNLEVELWLITAFHDKGSLTDYLKGNIITWNELCHVAETMSRGLSYLHEDVPWCRGEGHKPSIAHRDFKSKNVLLKSDLTAVLADFGLAVRFEPGKPPGDTHGQVGTRRYMAPEVLEGAINFQRDAFLRIDMYAMGLVLWELVSRCKAADGPVDEYMLPFEEEIGQHPSLEELQEVVVHKKMRPTIKDHWLKHPGLAQLCVTIEECWDHDAEARLSAGCVEERVSLIRRSVNGTTSDCLVSLVTSVTNVDLPPKESSI",
# COQ8B_HUMAN
"MWLKVGGLLRGTGGQLGQTVGWPCGALGPGPHRWGPCGGSWAQKFYQDGPGRGLGEEDIRRAREARPRKTPRPQLSDRSRERKVPASRISRLANFGGLAVGLGLGVLAEMAKKSMPGGRLQSEGGSGLDSSPFLSEANAERIVQTLCTVRGAALKVGQMLSIQDNSFISPQLQHIFERVRQSADFMPRWQMLRVLEEELGRDWQAKVASLEEVPFAAASIGQVHQGLLRDGTEVAVKIQYPGIAQSIQSDVQNLLAVLKMSAALPAGLFAEQSLQALQQELAWECDYRREAACAQNFRQLLANDPFFRVPAVVKELCTTRVLGMELAGGVPLDQCQGLSQDLRNQICFQLLTLCLRELFEFRFMQTDPNWANFLYDASSHQVTLLDFGASREFGTEFTDHYIEVVKAAADGDRDCVLQKSRDLKFLTGFETKAFSDAHVEAVMILGEPFATQGPYDFGSGETARRIQDLIPVLLRHRLCPPPEETYALHRKLAGAFLACAHLRAHIACRDLFQDTYHRYWASRQPDAATAGSLPTKGDSWVDPS",
# TRIB1_HUMAN
"MRVGPVRSAMSGASQPRGPALLFPATRGVPAKRLLDADDAAAVAAKCPRLSECSSPPDYLSPPGSPCSPQPPPAAPGAGGGSGSAPGPSRIADYLLLPLAEREHVSRALCIHTGRELRCKVFPIKHYQDKIRPYIQLPSHSNITGIVEVILGETKAYVFFEKDFGDMHSYVRSRKRLREEEAARLFKQIVSAVAHCHQSAIVLGDLKLRKFVFSTEERTQLRLESLEDTHIMKGEDDALSDKHGCPAYVSPEILNTTGTYSGKAADVWSLGVMLYTLLVGRYPFHDSDPSALFSKIRRGQFCIPEHISPKARCLIRSLLRREPSERLTAPEILLHPWFESVLEPGYIDSEIGTSDQIVPEYQEDSDISSFFC",
# BRSK2_HUMAN
"MTSTGKDGGAQHAQYVGPYRLEKTLGKGQTGLVKLGVHCVTCQKVAIKIVNREKLSESVLMKVEREIAILKLIEHPHVLKLHDVYENKKYLYLVLEHVSGGELFDYLVKKGRLTPKEARKFFRQIISALDFCHSHSICHRDLKPENLLLDEKNNIRIADFGMASLQVGDSLLETSCGSPHYACPEVIRGEKYDGRKADVWSCGVILFALLVGALPFDDDNLRQLLEKVKRGVFHMPHFIPPDCQSLLRGMIEVDAARRLTLEHIQKHIWYIGGKNEPEPEQPIPRKVQIRSLPSLEDIDPDVLDSMHSLGCFRDRNKLLQDLLSEEENQEKMIYFLLLDRKERYPSQEDEDLPPRNEIDPPRKRVDSPMLNRHGKRRPERKSMEVLSVTDGGSPVPARRAIEMAQHGQRSRSISGASSGLSTSPLSSPRVTPHPSPRGSPLPTPKGTPVHTPKESPAGTPNPTPPSSPSVGGVPWRARLNSIKNSFLGSPRFHRRKLQVPTPEEMSNLTPESSPELAKKSWFGNFISLEKEEQIFVVIKDKPLSSIKADIVHAFLSIPSLSHSVISQTSFRAEYKATGGPAVFQKPVKFQVDITYTEGGEAQKENGIYSVTFTLLSGPSRRFKRVVETIQAQLLSTHDPPAAQHLSDTTNCMEMMTGRLSKCGSPLSNFFDVIKQLFSDEKNGQAAQAPSTPAKRSAHGPLGDSAAAGPGPGGDAEYPTGKDTAKMGPPTARREQP",
# WNK2_HUMAN
"MDGDGGRRDVPGTLMEPGRGAGPAGMAEPRAKAARPGPQRFLRRSVVESDQEEPPGLEAAEAPGPQPPQPLQRRVLLLCKTRRLIAERARGRPAAPAPAALVAQPGAPGAPADAGPEPVGTQEPGPDPIAAAVETAPAPDGGPREEAAATVRKEDEGAAEAKPEPGRTRRDEPEEEEDDEDDLKAVATSLDGRFLKFDIELGRGSFKTVYKGLDTETWVEVAWCELQDRKLTKLERQRFKEEAEMLKGLQHPNIVRFYDFWESSAKGKRCIVLVTELMTSGTLKTYLKRFKVMKPKVLRSWCRQILKGLLFLHTRTPPIIHRDLKCDNIFITGPTGSVKIGDLGLATLKRASFAKSVIGTPEFMAPEMYEEHYDESVDVYAFGMCMLEMATSEYPYSECQNAAQIYRKVTCGIKPASFEKVHDPEIKEIIGECICKNKEERYEIKDLLSHAFFAEDTGVRVELAEEDHGRKSTIALRLWVEDPKKLKGKPKDNGAIEFTFDLEKETPDEVAQEMIESGFFHESDVKIVAKSIRDRVALIQWRRERIWPALQPKEQQDVGSPDKARGPPVPLQVQVTYHAQAGQPGPPEPEEPEADQHLLPPTLPTSATSLASDSTFDSGQGSTVYSDSQSSQQSVMLGSLADAAPSPAQCVCSPPVSEGPVLPQSLPSLGAYQQPTAAPGLPVGSVPAPACPPSLQQHFPDPAMSFAPVLPPPSTPMPTGPGQPAPPGQQPPPLAQPTPLPQVLAPQPVVPLQPVPPHLPPYLAPASQVGAPAQLKPLQMPQAPLQPLAQVPPQMPPIPVVPPITPLAGIDGLPPALPDLPTATVPPVPPPQYFSPAVILPSLAAPLPPASPALPLQAVKLPHPPGAPLAMPCRTIVPNAPATIPLLAVAPPGVAALSIHSAVAQLPGQPVYPAAFPQMAPTDVPPSPHHTVQNMRATPPQPALPPQPTLPPQPVLPPQPTLPPQPVLPPQPTRPPQPVLPPQPMLPPQPVLPPQPALPVRPEPLQPHLPEQAAPAATPGSQILLGHPAPYAVDVAAQVPTVPVPPAAVLSPPLPEVLLPAAPELLPQFPSSLATVSASVQSVPTQTATLLPPANPPLPGGPGIASPCPTVQLTVEPVQEEQASQDKPPGLPQSCESYGGSDVTSGKELSDSCEGAFGGGRLEGRAARKHHRRSTRARSRQERASRPRLTILNVCNTGDKMVECQLETHNHKMVTFKFDLDGDAPDEIATYMVEHDFILQAERETFIEQMKDVMDKAEDMLSEDTDADRGSDPGTSPPHLSTCGLGTGEESRQSQANAPVYQQNVLHTGKRWFIICPVAEHPAPEAPESSPPLPLSSLPPEASQGPCRGLTLPCLPWRRAACGAVFLSLFSAESAQSKQPPDSAPYKDQLSSKEQPSFLASQQLLSQAGPSNPPGAPPAPLAPSSPPVTALPQDGAAPATSTMPEPASGTASQAGGPGTPQGLTSELETSQPLAETHEAPLAVQPLVVGLAPCTPAPEAASTRDASAPREPLPPPAPEPSPHSGTPQPALGQPAPLLPAAVGAVSLATSQLPSPPLGPTVPPQPPSALESDGEGPPPRVGFVDSTIKSLDEKLRTLLYQEHVPTSSASAGTPVEVGDRDFTLEPLRGDQPRSEVCGGDLALPPVPKEAVSGRVQLPQPLVEKSELAPTRGAVMEQGTSSSMTAESSPRSMLGYDRDGRQVASDSHVVPSVPQDVPAFVRPARVEPTDRDGGEAGESSAEPPPSDMGTVGGQASHPQTLGARALGSPRKRPEQQDVSSPAKTVGRFSVVSTQDEWTLASPHSLRYSAPPDVYLDEAPSSPDVKLAVRRAQTASSIEVGVGEPVSSDSGDEGPRARPPVQKQASLPVSGSVAGDFVKKATAFLQRPSRAGSLGPETPSRVGMKVPTISVTSFHSQSSYISSDNDSELEDADIKKELQSLREKHLKEISELQSQQKQEIEALYRRLGKPLPPNVGFFHTAPPTGRRRKTSKSKLKAGKLLNPLVRQLKVVASSTGHLADSSRGPPAKDPAQASVGLTADSTGLSGKAVQTQQPCSVRASLSSDICSGLASDGGGARGQGWTVYHPTSERVTYKSSSKPRARFLSGPVSVSIWSALKRLCLGKEHSSRSSTSSLAPGPEPGPQPALHVQAQVNNSNNKKGTFTDDLHKLVDEWTSKTVGAAQLKPTLNQLKQTQKLQDMEAQAGWAAPGEARAMTAPRAGVGMPRLPPAPGPLSTTVIPGAAPTLSVPTPDGALGTARRNQVWFGLRVPPTACCGHSTQPRGGQRVGSKTASFAASDPVRS",
# AKT1_HUMAN
"MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVDQREAPLNNFSVAQCQLMKTERPRPNTFIIRCLQWTTVIERTFHVETPEEREEWTTAIQTVADGLKKQEEEEMDFRSGSPSDNSGAEEMEVSLAKPKHRVTMNEFEYLKLLGKGTFGKVILVKEKATGRYYAMKILKKEVIVAKDEVAHTLTENRVLQNSRHPFLTALKYSFQTHDRLCFVMEYANGGELFFHLSRERVFSEDRARFYGAEIVSALDYLHSEKNVVYRDLKLENLMLDKDGHIKITDFGLCKEGIKDGATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHEKLFELILMEEIRFPRTLGPEAKSLLSGLLKKDPKQRLGGGSEDAKEIMQHRFFAGIVWQHVYEKKLSPPFKPQVTSETDTRYFDEEFTAQMITITPPDQDDSMECVDSERRPHFPQFSYSASGTA",
# AKT2_HUMAN
"MNEVSVIKEGWLHKRGEYIKTWRPRYFLLKSDGSFIGYKERPEAPDQTLPPLNNFSVAECQLMKTERPRPNTFVIRCLQWTTVIERTFHVDSPDEREEWMRAIQMVANSLKQRAPGEDPMDYKCGSPSDSSTTEEMEVAVSKARAKVTMNDFDYLKLLGKGTFGKVILVREKATGRYYAMKILRKEVIIAKDEVAHTVTESRVLQNTRHPFLTALKYAFQTHDRLCFVMEYANGGELFFHLSRERVFTEERARFYGAEIVSALEYLHSRDVVYRDIKLENLMLDKDGHIKITDFGLCKEGISDGATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHERLFELILMEEIRFPRTLSPEAKSLLAGLLKKDPKQRLGGGPSDAKEVMEHRFFLSINWQDVVQKKLLPPFKPQVTSEVDTRYFDDEFTAQSITITPPDRYDSLGLLELDQRTHFPQFSYSASIRE",
# AKT3_HUMAN
"MSDVTIVKEGWVQKRGEYIKNWRPRYFLLKTDGSFIGYKEKPQDVDLPYPLNNFSVAKCQLMKTERPKPNTFIIRCLQWTTVIERTFHVDTPEEREEWTEAIQAVADRLQRQEEERMNCSPTSQIDNIGEEEMDASTTHHKRKTMNDFDYLKLLGKGTFGKVILVREKASGKYYAMKILKKEVIIAKDEVAHTLTESRVLKNTRHPFLTSLKYSFQTKDRLCFVMEYVNGGELFFHLSRERVFSEDRTRFYGAEIVSALDYLHSGKIVYRDLKLENLMLDKDGHIKITDFGLCKEGITDAATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHEKLFELILMEDIKFPRTLSSDAKSLLSGLLIKDPNKRLGGGPDDAKEIMRHSFFSGVNWQDVYDKKLVPPFKPQVTSETDTRYFDEEFTAQTITITPPEKYDEDGMDCMDNERRPHFPQFSYSASGRE",
# KCC1G_HUMAN
"MGRKEEDDCSSWKKQTTNIRKTFIFMEVLGSGAFSEVFLVKQRLTGKLFALKCIKKSPAFRDSSLENEIAVLKKIKHENIVTLEDIYESTTHYYLVMQLVSGGELFDRILERGVYTEKDASLVIQQVLSAVKYLHENGIVHRDLKPENLLYLTPEENSKIMITDFGLSKMEQNGIMSTACGTPGYVAPEVLAQKPYSKAVDCWSIGVITYILLCGYPPFYEETESKLFEKIKEGYYEFESPFWDDISESAKDFICHLLEKDPNERYTCEKALSHPWIDGNTALHRDIYPSVSLQIQKNFAKSKWRQAFNAAAVVHHMRKLHMNLHSPGVRPEVENRPPETQASETSRPSSPEITITEAPVLDHSVALPALTQLPCQHGRRPTAPGGRSLNCLVNGSLHISSSLVPMHQGSLAAGPCGCCSSCLNIGSKGKSSYCSEPTLLKKANKKQNFKSEVMVPVKASGSSHCRAGQTGVCLIM",
# ALK_HUMAN
"MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSYSRLQRKSLAVDFVVPSLFRVYARDLLLPPSSSELKAGRPEARGSLALDCAPLLRLLGPAPGVSWTAGSPAPAEARTLSRVLKGGSVRKLRRAKQLVLELGEEAILEGCVGPPGEAAVGLLQFNLSELFSWWIRQGEGRLRIRLMPEKKASEVGREGRLSAAIRASQPRLLFQIFGTGHSSLESPTNMPSPSPDYFTWNLTWIMKDSFPFLSHRSRYGLECSFDFPCELEYSPPLHDLRNQSWSWRRIPSEEASQMDLLDGPGAERSKEMPRGSFLLLNTSADSKHTILSPWMRSSSEHCTLAVSVHRHLQPSGRYIAQLLPHNEAAREILLMPTPGKHGWTVLQGRIGRPDNPFRVALEYISSGNRSLSAVDFFALKNCSEGTSPGSKMALQSSFTCWNGTVLQLGQACDFHQDCAQGEDESQMCRKLPVGFYCNFEDGFCGWTQGTLSPHTPQWQVRTLKDARFQDHQDHALLLSTTDVPASESATVTSATFPAPIKSSPCELRMSWLIRGVLRGNVSLVLVENKTGKEQGRMVWHVAAYEGLSLWQWMVLPLLDVSDRFWLQMVAWWGQGSRAIVAFDNISISLDCYLTISGEDKILQNTAPKSRNLFERNPNKELKPGENSPRQTPIFDPTVHWLFTTCGASGPHGPTQAQCNNAYQNSNLSVEVGSEGPLKGIQIWKVPATDTYSISGYGAAGGKGGKNTMMRSHGVSVLGIFNLEKDDMLYILVGQQGEDACPSTNQLIQKVCIGENNVIEEEIRVNRSVHEWAGGGGGGGGATYVFKMKDGVPVPLIIAAGGGGRAYGAKTDTFHPERLENNSSVLGLNGNSGAAGGGGGWNDNTSLLWAGKSLQEGATGGHSCPQAMKKWGWETRGGFGGGGGGCSSGGGGGGYIGGNAASNNDPEMDGEDGVSFISPLGILYTPALKVMEGHGEVNIKHYLNCSHCEVDECHMDPESHKVICFCDHGTVLAEDGVSCIVSPTPEPHLPLSLILSVVTSALVAALVLAFSGIMIVYRRKHQELQAMQMELQSPEYKLSKLRTSTIMTDYNPNYCFAGKTSSISDLKEVPRKNITLIRGLGHGAFGEVYEGQVSGMPNDPSPLQVAVKTLPEVCSEQDELDFLMEALIISKFNHQNIVRCIGVSLQSLPRFILLELMAGGDLKSFLRETRPRPSQPSSLAMLDLLHVARDIACGCQYLEENHFIHRDIAARNCLLTCPGPGRVAKIGDFGMARDIYRASYYRKGGCAMLPVKWMPPEAFMEGIFTSKTDTWSFGVLLWEIFSLGYMPYPSKSNQEVLEFVTSGGRMDPPKNCPGPVYRIMTQCWQHQPEDRPNFAIILERIEYCTQDPDVINTALPIEYGPLVEEEEKVPVRPKDPEGVPPLLVSQQAKREEERSPAAPPPLPTTSSGKAAKKPTAAEISVRVPRGPAVEGGHVNMAFSQSNPPSELHKVHGSRNKPTSLWNPTYGSWFTEKPTKKNNPIAKKEPHDRGNLGLEGSCTVPPNVATGRLPGASLLLEPSSLTANMKEVPLFRLRHFPCGNVNYGYQQQGLPLEAATAPGAGHYEDTILKSKNSMNQPGP",
# ACVL1_HUMAN
"MTLGSPRKGLLMLLMALVTQGDPVKPSRGPLVTCTCESPHCKGPTCRGAWCTVVLVREEGRHPQEHRGCGNLHRELCRGRPTEFVNHYCCDSHLCNHNVSLVLEATQPPSEQPGTDGQLALILGPVLALLALVALGVLGLWHVRRRQEKQRGLHSELGESSLILKASEQGDSMLGDLLDSDCTTGSGSGLPFLVQRTVARQVALVECVGKGRYGEVWRGLWHGESVAVKIFSSRDEQSWFRETEIYNTVLLRHDNILGFIASDMTSRNSSTQLWLITHYHEHGSLYDFLQRQTLEPHLALRLAVSAACGLAHLHVEIFGTQGKPAIAHRDFKSRNVLVKSNLQCCIADLGLAVMHSQGSDYLDIGNNPRVGTKRYMAPEVLDEQIRTDCFESYKWTDIWAFGLVLWEIARRTIVNGIVEDYRPPFYDVVPNDPSFEDMKKVVCVDQQTPTIPNRLAADPVLSGLAQMMRECWYPNPSARLTALRIKKTLQKISNSPEKPKVIQ",
# ACVR1_HUMAN
"MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNEDHCEGQQCFSSLSINDGFHVYQKGCFQVYEQGKMTCKTPPSPGQAVECCQGDWCNRNITAQLPTKGKSFPGTQNFHLEVGLIILSVVFAVCLLACLLGVALRKFKRRNQERLNPRDVEYGTIEGLITTNVGDSTLADLLDHSCTSGSGSGLPFLVQRTVARQITLLECVGKGRYGEVWRGSWQGENVAVKIFSSRDEKSWFRETELYNTVMLRHENILGFIASDMTSRHSSTQLWLITHYHEMGSLYDYLQLTTLDTVSCLRIVLSIASGLAHLHIEIFGTQGKPAIAHRDLKSKNILVKKNGQCCIADLGLAVMHSQSTNQLDVGNNPRVGTKRYMAPEVLDETIQVDCFDSYKRVDIWAFGLVLWEVARRMVSNGIVEDYKPPFYDVVPNDPSFEDMRKVVCVDQQRPNIPNRWFSDPTLTSLAKLMKECWYQNPSARLTALRIKKTLTKIDNSLDKLKTDC",
# BMR1A_HUMAN
"MPQLYIYIRLLGAYLFIISRVQGQNLDSMLHGTGMKSDSDQKKSENGVTLAPEDTLPFLKCYCSGHCPDDAINNTCITNGHCFAIIEEDDQGETTLASGCMKYEGSDFQCKDSPKAQLRRTIECCRTNLCNQYLQPTLPPVVIGPFFDGSIRWLVLLISMAVCIIAMIIFSSCFCYKHYCKSISSRRRYNRDLEQDEAFIPVGESLKDLIDQSQSSGSGSGLPLLVQRTIAKQIQMVRQVGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQTVLMRHENILGFIAADIKGTGSWTQLYLITDYHENGSLYDFLKCATLDTRALLKLAYSAACGLCHLHTEIYGTQGKPAIAHRDLKSKNILIKKNGSCCIADLGLAVKFNSDTNEVDVPLNTRVGTKRYMAPEVLDESLNKNHFQPYIMADIYSFGLIIWEMARRCITGGIVEEYQLPYYNMVPSDPSYEDMREVVCVKRLRPIVSNRWNSDECLRAVLKLMSECWAHNPASRLTALRIKKTLAKMVESQDVKI",
# ACV1B_HUMAN
"MAESAGASSFFPLVVLLLAGSGGSGPRGVQALLCACTSCLQANYTCETDGACMVSIFNLDGMEHHVRTCIPKVELVPAGKPFYCLSSEDLRNTHCCYTDYCNRIDLRVPSGHLKEPEHPSMWGPVELVGIIAGPVFLLFLIIIIVFLVINYHQRVYHNRQRLDMEDPSCEMCLSKDKTLQDLVYDLSTSGSGSGLPLFVQRTVARTIVLQEIIGKGRFGEVWRGRWRGGDVAVKIFSSREERSWFREAEIYQTVMLRHENILGFIAADNKDNGTWTQLWLVSDYHEHGSLFDYLNRYTVTIEGMIKLALSAASGLAHLHMEIVGTQGKPGIAHRDLKSKNILVKKNGMCAIADLGLAVRHDAVTDTIDIAPNQRVGTKRYMAPEVLDETINMKHFDSFKCADIYALGLVYWEIARRCNSGGVHEEYQLPYYDLVPSDPSIEEMRKVVCDQKLRPNIPNWWQSYEALRVMGKMMRECWYANGAARLTALRIKKTLSQLSVQEDVKI",
# TGFR1_HUMAN
"MEAAVAAPRPRLLLLVLAAAAAAAAALLPGATALQCFCHLCTKDNFTCVTDGLCFVSVTETTDKVIHNSMCIAEIDLIPRDRPFVCAPSSKTGSVTTTYCCNQDHCNKIELPTTVKSSPGLGPVELAAVIAGPVCFVCISLMLMVYICHNRTVIHHRVPNEEDPSLDRPFISEGTTLKDLIYDMTTSGSGSGLPLLVQRTIARTIVLQESIGKGRFGEVWRGKWRGEEVAVKIFSSREERSWFREAEIYQTVMLRHENILGFIAADNKDNGTWTQLWLVSDYHEHGSLFDYLNRYTVTVEGMIKLALSTASGLAHLHMEIVGTQGKPAIAHRDLKSKNILVKKNGTCCIADLGLAVRHDSATDTIDIAPNHRVGTKRYMAPEVLDDSINMKHFESFKRADIYAMGLVFWEIARRCSIGGIHEDYQLPYYDLVPSDPSVEEMRKVVCEQKLRPNIPNRWQSCEALRVMAKIMRECWYANGAARLTALRIKKTLSQLSQQEGIKM",
# BMR1B_HUMAN
"MLLRSAGKLNVGTKKEDGESTAPTPRPKVLRCKCHHHCPEDSVNNICSTDGYCFTMIEEDDSGLPVVTSGCLGLEGSDFQCRDTPIPHQRRSIECCTERNECNKDLHPTLPPLKNRDFVDGPIHHRALLISVTVCSLLLVLIILFCYFRYKRQETRPRYSIGLEQDETYIPPGESLRDLIEQSQSSGSGSGLPLLVQRTIAKQIQMVKQIGKGRYGEVWMGKWRGEKVAVKVFFTTEEASWFRETEIYQTVLMRHENILGFIAADIKGTGSWTQLYLITDYHENGSLYDYLKSTTLDAKSMLKLAYSSVSGLCHLHTEIFSTQGKPAIAHRDLKSKNILVKKNGTCCIADLGLAVKFISDTNEVDIPPNTRVGTKRYMPPEVLDESLNRNHFQSYIMADMYSFGLILWEVARRCVSGGIVEEYQLPYHDLVPSDPSYEDMREIVCIKKLRPSFPNRWSSDECLRQMGKLMTECWAHNPASRLTALRVKKTLAKMSESQDIKL",
# AAPK1_HUMAN
"MRRLSSWRKMATAEKQKHDGRVKIGHYILGDTLGVGTFGKVKVGKHELTGHKVAVKILNRQKIRSLDVVGKIRREIQNLKLFRHPHIIKLYQVISTPSDIFMVMEYVSGGELFDYICKNGRLDEKESRRLFQQILSGVDYCHRHMVVHRDLKPENVLLDAHMNAKIADFGLSNMMSDGEFLRTSCGSPNYAAPEVISGRLYAGPEVDIWSSGVILYALLCGTLPFDDDHVPTLFKKICDGIFYTPQYLNPSVISLLKHMLQVDPMKRATIKDIREHEWFKQDLPKYLFPEDPSYSSTMIDDEALKEVCEKFECSEEEVLSCLYNRNHQDPLAVAYHLIIDNRRIMNEAKDFYLATSPPDSFLDDHHLTRPHPERVPFLVAETPRARHTLDELNPQKSKHQGVRKAKWHLGIRSQSRPNDIMAEVCRAIKQLDYEWKVVNPYYLRVRRKNPVTSTYSKMSLQLYQVDSRTYLLDFRSIDDEITEAKSGTATPQRSGSVSNYRSCQRSDSDAEAQGKSSEVSLTSSVTSLDSSPVDLTPRPGSHTIEFFEMCANLIKILAQ",
# AAPK2_HUMAN
"MAEKQKHDGRVKIGHYVLGDTLGVGTFGKVKIGEHQLTGHKVAVKILNRQKIRSLDVVGKIKREIQNLKLFRHPHIIKLYQVISTPTDFFMVMEYVSGGELFDYICKHGRVEEMEARRLFQQILSAVDYCHRHMVVHRDLKPENVLLDAHMNAKIADFGLSNMMSDGEFLRTSCGSPNYAAPEVISGRLYAGPEVDIWSCGVILYALLCGTLPFDDEHVPTLFKKIRGGVFYIPEYLNRSVATLLMHMLQVDPLKRATIKDIREHEWFKQDLPSYLFPEDPSYDANVIDDEAVKEVCEKFECTESEVMNSLYSGDPQDQLAVAYHLIIDNRRIMNQASEFYLASSPPSGSFMDDSAMHIPPGLKPHPERMPPLIADSPKARCPLDALNTTKPKSLAVKKAKWHLGIRSQSKPYDIMAEVYRAMKQLDFEWKVVNAYHLRVRRKNPVTGNYVKMSLQLYLVDNRSYLLDFKSIDDEVVEQRSGSSTPQRSCSAAGLHRPRSSFDSTTAESHSLSGSLTGSLTGSTLSSVSPRLGSHTMDFFEMCASLITTLAR",
# ANPRA_HUMAN
"MPGPRRPAGSRLRLLLLLLLPPLLLLLRGSHAGNLTVAVVLPLANTSYPWSWARVGPAVELALAQVKARPDLLPGWTVRTVLGSSENALGVCSDTAAPLAAVDLKWEHNPAVFLGPGCVYAAAPVGRFTAHWRVPLLTAGAPALGFGVKDEYALTTRAGPSYAKLGDFVAALHRRLGWERQALMLYAYRPGDEEHCFFLVEGLFMRVRDRLNITVDHLEFAEDDLSHYTRLLRTMPRKGRVIYICSSPDAFRTLMLLALEAGLCGEDYVFFHLDIFGQSLQGGQGPAPRRPWERGDGQDVSARQAFQAAKIITYKDPDNPEYLEFLKQLKHLAYEQFNFTMEDGLVNTIPASFHDGLLLYIQAVTETLAHGGTVTDGENITQRMWNRSFQGVTGYLKIDSSGDRETDFSLWDMDPENGAFRVVLNYNGTSQELVAVSGRKLNWPLGYPPPDIPKCGFDNEDPACNQDHLSTLEVLALVGSLSLLGILIVSFFIYRKMQLEKELASELWRVRWEDVEPSSLERHLRSAGSRLTLSGRGSNYGSLLTTEGQFQVFAKTAYYKGNLVAVKRVNRKRIELTRKVLFELKHMRDVQNEHLTRFVGACTDPPNICILTEYCPRGSLQDILENESITLDWMFRYSLTNDIVKGMLFLHNGAICSHGNLKSSNCVVDGRFVLKITDYGLESFRDLDPEQGHTVYAKKLWTAPELLRMASPPVRGSQAGDVYSFGIILQEIALRSGVFHVEGLDLSPKEIIERVTRGEQPPFRPSLALQSHLEELGLLMQRCWAEDPQERPPFQQIRLTLRKFNRENSSNILDNLLSRMEQYANNLEELVEERTQAYLEEKRKAEALLYQILPHSVAEQLKRGETVQAEAFDSVTIYFSDIVGFTALSAESTPMQVVTLLNDLYTCFDAVIDNFDVYKVETIGDAYMVVSGLPVRNGRLHACEVARMALALLDAVRSFRIRHRPQEQLRLRIGIHTGPVCAGVVGLKMPRYCLFGDTVNTASRMESNGEALKIHLSSETKAVLEEFGGFELELRGDVEMKGKGKVRTYWLLGERGSSTRG",
# ANPRB_HUMAN
"MALPSLLLLVAALAGGVRPPGARNLTLAVVLPEHNLSYAWAWPRVGPAVALAVEALGRALPVDLRFVSSELEGACSEYLAPLSAVDLKLYHDPDLLLGPGCVYPAASVARFASHWRLPLLTAGAVASGFSAKNDHYRTLVRTGPSAPKLGEFVVTLHGHFNWTARAALLYLDARTDDRPHYFTIEGVFEALQGSNLSVQHQVYAREPGGPEQATHFIRANGRIVYICGPLEMLHEILLQAQRENLTNGDYVFFYLDVFGESLRAGPTRATGRPWQDNRTREQAQALREAFQTVLVITYREPPNPEYQEFQNRLLIRAREDFGVELGPSLMNLIAGCFYDGILLYAEVLNETIQEGGTREDGLRIVEKMQGRRYHGVTGLVVMDKNNDRETDFVLWAMGDLDSGDFQPAAHYSGAEKQIWWTGRPIPWVKGAPPSDNPPCAFDLDDPSCDKTPLSTLAIVALGTGITFIMFGVSSFLIFRKLMLEKELASMLWRIRWEELQFGNSERYHKGAGSRLTLSLRGSSYGSLMTAHGKYQIFANTGHFKGNVVAIKHVNKKRIELTRQVLFELKHMRDVQFNHLTRFIGACIDPPNICIVTEYCPRGSLQDILENDSINLDWMFRYSLINDLVKGMAFLHNSIISSHGSLKSSNCVVDSRFVLKITDYGLASFRSTAEPDDSHALYAKKLWTAPELLSGNPLPTTGMQKADVYSFGIILQEIALRSGPFYLEGLDLSPKEIVQKVRNGQRPYFRPSIDRTQLNEELVLLMERCWAQDPAERPDFGQIKGFIRRFNKEGGTSILDNLLLRMEQYANNLEKLVEERTQAYLEEKRKAEALLYQILPHSVAEQLKRGETVQAEAFDSVTIYFSDIVGFTALSAESTPMQVVTLLNDLYTCFDAIIDNFDVYKVETIGDAYMVVSGLPGRNGQRHAPEIARMALALLDAVSSFRIRHRPHDQLRLRIGVHTGPVCAGVVGLKMPRYCLFGDTVNTASRMESNGQALKIHVSSTTKDALDELGCFQLELRGDVEMKGKGKMRTYWLLGERKGPPGLL",
# ARAF_HUMAN
"MEPPRGPPANGAEPSRAVGTVKVYLPNKQRTVVTVRDGMSVYDSLDKALKVRGLNQDCCVVYRLIKGRKTVTAWDTAIAPLDGEELIVEVLEDVPLTMHNFVRKTFFSLAFCDFCLKFLFHGFRCQTCGYKFHQHCSSKVPTVCVDMSTNRQQFYHSVQDLSGGSRQHEAPSNRPLNELLTPQGPSPRTQHCDPEHFPFPAPANAPLQRIRSTSTPNVHMVSTTAPMDSNLIQLTGQSFSTDAAGSRGGSDGTPRGSPSPASVSSGRKSPHSKSPAEQRERKSLADDKKKVKNLGYRDSGYYWEVPPSEVQLLKRIGTGSFGTVFRGRWHGDVAVKVLKVSQPTAEQAQAFKNEMQVLRKTRHVNILLFMGFMTRPGFAIITQWCEGSSLYHHLHVADTRFDMVQLIDVARQTAQGMDYLHAKNIIHRDLKSNNIFLHEGLTVKIGDFGLATVKTRWSGAQPLEQPSGSVLWMAAEVIRMQDPNPYSFQSDVYAYGVVLYELMTGSLPYSHIGCRDQIIFMVGRGYLSPDLSKISSNCPKAMRRLLSDCLKFQREERPLFPQILATIELLQRSLPKIERSASEPSLHRTQADELPACLLSAARLVP",
# ABL2_HUMAN
"MGQQVGRVGEAPGLQQPQPRGIRGSSAARPSGRRRDPAGRTTETGFNIFTQHDHFASCVEDGFEGDKTGGSSPEALHRPYGCDVEPQALNEAIRWSSKENLLGATESDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNQNGEWSEVRSKNGQGWVPSNYITPVNSLEKHSWYHGPVSRSAAEYLLSSLINGSFLVRESESSPGQLSISLRYEGRVYHYRINTTADGKVYVTAESRFSTLAELVHHHSTVADGLVTTLHYPAPKCNKPTVYGVSPIHDKWEMERTDITMKHKLGGGQYGEVYVGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTLEPPFYIVTEYMPYGNLLDYLRECNREEVTAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHVVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNTFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYDLLEKGYRMEQPEGCPPKVYELMRACWKWSPADRPSFAETHQAFETMFHDSSISEEVAEELGRAASSSSVVPYLPRLPILPSKTRTLKKQVENKENIEGAQDATENSASSLAPGFIRGAQASSGSPALPRKQRDKSPSSLLEDAKETCFTRDRKGGFFSSFMKKRNAPTPPKRSSSFREMENQPHKKYELTGNFSSVASLQHADGFSFTPAQQEANLVPPKCYGGSFAQRNLCNDDGGGGGGSGTAGGGWSGITGFFTPRLIKKTLGLRAGKPTASDDTSKPFPRSNSTSSMSSGLPEQDRMAMTLPRNCQRSKLQLERTVSTSSQPEENVDRANDMLPKKSEESAAPSRERPKAKLLPRGATALPLRTPSGDLAITEKDPPGVGVAGVAAAPKGKEKNGGARLGMAGVPEDGEQPGWPSPAKAAPVLPTTHNHKVPVLISPTLKHTPADVQLIGTDSQGNKFKLLSEHQVTSSGDKDRPRRVKPKCAPPPPPVMRLLQHPSICSDPTEEPTALTAGQSTSETQEGGKKAALGAVPISGKAGRPVMPPPQVPLPTSSISPAKMANGTAGTKVALRKTKQAAEKISADKISKEALLECADLLSSALTEPVPNSQLVDTGHQLLDYCSGYVDCIPQTRNKFAFREAVSKLELSLQELQVSSAAAGVPGTNPVLNNLLSCVQEISDVVQR",
# ATM_HUMAN
"MSLVLNDLLICCRQLEHDRATERKKEVEKFKRLIRDPETIKHLDRHSDSKQGKYLNWDAVFRFLQKYIQKETECLRIAKPNVSASTQASRQKKMQEISSLVKYFIKCANRRAPRLKCQELLNYIMDTVKDSSNGAIYGADCSNILLKDILSVRKYWCEISQQQWLELFSVYFRLYLKPSQDVHRVLVARIIHAVTKGCCSQTDGLNSKFLDFFSKAIQCARQEKSSSGLNHILAALTIFLKTLAVNFRIRVCELGDEILPTLLYIWTQHRLNDSLKEVIIELFQLQIYIHHPKGAKTQEKGAYESTKWRSILYNLYDLLVNEISHIGSRGKYSSGFRNIAVKENLIELMADICHQVFNEDTRSLEISQSYTTTQRESSDYSVPCKRKKIELGWEVIKDHLQKSQNDFDLVPWLQIATQLISKYPASLPNCELSPLLMILSQLLPQQRHGERTPYVLRCLTEVALCQDKRSNLESSQKSDLLKLWNKIWCITFRGISSEQIQAENFGLLGAIIQGSLVEVDREFWKLFTGSACRPSCPAVCCLTLALTTSIVPGTVKMGIEQNMCEVNRSFSLKESIMKWLLFYQLEGDLENSTEVPPILHSNFPHLVLEKILVSLTMKNCKAAMNFFQSVPECEHHQKDKEELSFSEVEELFLQTTFDKMDFLTIVRECGIEKHQSSIGFSVHQNLKESLDRCLLGLSEQLLNNYSSEITNSETLVRCSRLLVGVLGCYCYMGVIAEEEAYKSELFQKAKSLMQCAGESITLFKNKTNEEFRIGSLRNMMQLCTRCLSNCTKKSPNKIASGFFLRLLTSKLMNDIADICKSLASFIKKPFDRGEVESMEDDTNGNLMEVEDQSSMNLFNDYPDSSVSDANEPGESQSTIGAINPLAEEYLSKQDLLFLDMLKFLCLCVTTAQTNTVSFRAADIRRKLLMLIDSSTLEPTKSLHLHMYLMLLKELPGEEYPLPMEDVLELLKPLSNVCSLYRRDQDVCKTILNHVLHVVKNLGQSNMDSENTRDAQGQFLTVIGAFWHLTKERKYIFSVRMALVNCLKTLLEADPYSKWAILNVMGKDFPVNEVFTQFLADNHHQVRMLAAESINRLFQDTKGDSSRLLKALPLKLQQTAFENAYLKAQEGMREMSHSAENPETLDEIYNRKSVLLTLIAVVLSCSPICEKQALFALCKSVKENGLEPHLVKKVLEKVSETFGYRRLEDFMASHLDYLVLEWLNLQDTEYNLSSFPFILLNYTNIEDFYRSCYKVLIPHLVIRSHFDEVKSIANQIQEDWKSLLTDCFPKILVNILPYFAYEGTRDSGMAQQRETATKVYDMLKSENLLGKQIDHLFISNLPEIVVELLMTLHEPANSSASQSTDLCDFSGDLDPAPNPPHFPSHVIKATFAYISNCHKTKLKSILEILSKSPDSYQKILLAICEQAAETNNVYKKHRILKIYHLFVSLLLKDIKSGLGGAWAFVLRDVIYTLIHYINQRPSCIMDVSLRSFSLCCDLLSQVCQTAVTYCKDALENHLHVIVGTLIPLVYEQVEVQKQVLDLLKYLVIDNKDNENLYITIKLLDPFPDHVVFKDLRITQQKIKYSRGPFSLLEEINHFLSVSVYDALPLTRLEGLKDLRRQLELHKDQMVDIMRASQDNPQDGIMVKLVVNLLQLSKMAINHTGEKEVLEAVGSCLGEVGPIDFSTIAIQHSKDASYTKALKLFEDKELQWTFIMLTYLNNTLVEDCVKVRSAAVTCLKNILATKTGHSFWEIYKMTTDPMLAYLQPFRTSRKKFLEVPRFDKENPFEGLDDINLWIPLSENHDIWIKTLTCAFLDSGGTKCEILQLLKPMCEVKTDFCQTVLPYLIHDILLQDTNESWRNLLSTHVQGFFTSCLRHFSQTSRSTTPANLDSESEHFFRCCLDKKSQRTMLAVVDYMRRQKRPSSGTIFNDAFWLDLNYLEVAKVAQSCAAHFTALLYAEIYADKKSMDDQEKRSLAFEEGSQSTTISSLSEKSKEETGISLQDLLLEIYRSIGEPDSLYGCGGGKMLQPITRLRTYEHEAMWGKALVTYDLETAIPSSTRQAGIIQALQNLGLCHILSVYLKGLDYENKDWCPELEELHYQAAWRNMQWDHCTSVSKEVEGTSYHESLYNALQSLRDREFSTFYESLKYARVKEVEEMCKRSLESVYSLYPTLSRLQAIGELESIGELFSRSVTHRQLSEVYIKWQKHSQLLKDSDFSFQEPIMALRTVILEILMEKEMDNSQRECIKDILTKHLVELSILARTFKNTQLPERAIFQIKQYNSVSCGVSEWQLEEAQVFWAKKEQSLALSILKQMIKKLDASCAANNPSLKLTYTECLRVCGNWLAETCLENPAVIMQTYLEKAVEVAGNYDGESSDELRNGKMKAFLSLARFSDTQYQRIENYMKSSEFENKQALLKRAKEEVGLLREHKIQTNRYTVKVQRELELDELALRALKEDRKRFLCKAVENYINCLLSGEEHDMWVFRLCSLWLENSGVSEVNGMMKRDGMKIPTYKFLPLMYQLAARMGTKMMGGLGFHEVLNNLISRISMDHPHHTLFIILALANANRDEFLTKPEVARRSRITKNVPKQSSQLDEDRTEAANRIICTIRSRRPQMVRSVEALCDAYIILANLDATQWKTQRKGINIPADQPITKLKNLEDVVVPTMEIKVDHTGEYGNLVTIQSFKAEFRLAGGVNLPKIIDCVGSDGKERRQLVKGRDDLRQDAVMQQVFQMCNTLLQRNTETRKRKLTICTYKVVPLSQRSGVLEWCTGTVPIGEFLVNNEDGAHKRYRPNDFSAFQCQKKMMEVQKKSFEEKYEVFMDVCQNFQPVFRYFCMEKFLDPAIWFEKRLAYTRSVATSSIVGYILGLGDRHVQNILINEQSAELVHIDLGVAFEQGKILPTPETVPFRLTRDIVDGMGITGVEGVFRRCCEKTMEVMRNSQETLLTIVEVLLYDPLFDWTMNPLKALYLQQRPEDETELHPTLNADDQECKRNLSDIDQSFNKVAERVLMRLQEKLKGVEEGTVLSVGGQVNLLIQQAIDPKNLSRLFPGWKAWV",
# ATR_HUMAN
"MGEHGLELASMIPALRELGSATPEEYNTVVQKPRQILCQFIDRILTDVNVVAVELVKKTDSQPTSVMLLDFIQHIMKSSPLMFVNVSGSHEAKGSCIEFSNWIITRLLRIAATPSCHLLHKKICEVICSLLFLFKSKSPAIFGVLTKELLQLFEDLVYLHRRNVMGHAVEWPVVMSRFLSQLDEHMGYLQSAPLQLMSMQNLEFIEVTLLMVLTRIIAIVFFRRQELLLWQIGCVLLEYGSPKIKSLAISFLTELFQLGGLPAQPASTFFSSFLELLKHLVEMDTDQLKLYEEPLSKLIKTLFPFEAEAYRNIEPVYLNMLLEKLCVMFEDGVLMRLKSDLLKAALCHLLQYFLKFVPAGYESALQVRKVYVRNICKALLDVLGIEVDAEYLLGPLYAALKMESMEIIEEIQCQTQQENLSSNSDGISPKRRRLSSSLNPSKRAPKQTEEIKHVDMNQKSILWSALKQKAESLQISLEYSGLKNPVIEMLEGIAVVLQLTALCTVHCSHQNMNCRTFKDCQHKSKKKPSVVITWMSLDFYTKVLKSCRSLLESVQKLDLEATIDKVVKIYDALIYMQVNSSFEDHILEDLCGMLSLPWIYSHSDDGCLKLTTFAANLLTLSCRISDSYSPQAQSRCVFLLTLFPRRIFLEWRTAVYNWALQSSHEVIRASCVSGFFILLQQQNSCNRVPKILIDKVKDDSDIVKKEFASILGQLVCTLHGMFYLTSSLTEPFSEHGHVDLFCRNLKATSQHECSSSQLKASVCKPFLFLLKKKIPSPVKLAFIDNLHHLCKHLDFREDETDVKAVLGTLLNLMEDPDKDVRVAFSGNIKHILESLDSEDGFIKELFVLRMKEAYTHAQISRNNELKDTLILTTGDIGRAAKGDLVPFALLHLLHCLLSKSASVSGAAYTEIRALVAAKSVKLQSFFSQYKKPICQFLVESLHSSQMTALPNTPCQNADVRKQDVAHQREMALNTLSEIANVFDFPDLNRFLTRTLQVLLPDLAAKASPAASALIRTLGKQLNVNRREILINNFKYIFSHLVCSCSKDELERALHYLKNETEIELGSLLRQDFQGLHNELLLRIGEHYQQVFNGLSILASFASSDDPYQGPRDIISPELMADYLQPKLLGILAFFNMQLLSSSVGIEDKKMALNSLMSLMKLMGPKHVSSVRVKMMTTLRTGLRFKDDFPELCCRAWDCFVRCLDHACLGSLLSHVIVALLPLIHIQPKETAAIFHYLIIENRDAVQDFLHEIYFLPDHPELKKIKAVLQEYRKETSESTDLQTTLQLSMKAIQHENVDVRIHALTSLKETLYKNQEKLIKYATDSETVEPIISQLVTVLLKGCQDANSQARLLCGECLGELGAIDPGRLDFSTTETQGKDFTFVTGVEDSSFAYGLLMELTRAYLAYADNSRAQDSAAYAIQELLSIYDCREMETNGPGHQLWRRFPEHVREILEPHLNTRYKSSQKSTDWSGVKKPIYLSKLGSNFAEWSASWAGYLITKVRHDLASKIFTCCSIMMKHDFKVTIYLLPHILVYVLLGCNQEDQQEVYAEIMAVLKHDDQHTINTQDIASDLCQLSTQTVFSMLDHLTQWARHKFQALKAEKCPHSKSNRNKVDSMVSTVDYEDYQSVTRFLDLIPQDTLAVASFRSKAYTRAVMHFESFITEKKQNIQEHLGFLQKLYAAMHEPDGVAGVSAIRKAEPSLKEQILEHESLGLLRDATACYDRAIQLEPDQIIHYHGVVKSMLGLGQLSTVITQVNGVHANRSEWTDELNTYRVEAAWKLSQWDLVENYLAADGKSTTWSVRLGQLLLSAKKRDITAFYDSLKLVRAEQIVPLSAASFERGSYQRGYEYIVRLHMLCELEHSIKPLFQHSPGDSSQEDSLNWVARLEMTQNSYRAKEPILALRRALLSLNKRPDYNEMVGECWLQSARVARKAGHHQTAYNALLNAGESRLAELYVERAKWLWSKGDVHQALIVLQKGVELCFPENETPPEGKNMLIHGRAMLLVGRFMEETANFESNAIMKKYKDVTACLPEWEDGHFYLAKYYDKLMPMVTDNKMEKQGDLIRYIVLHFGRSLQYGNQFIYQSMPRMLTLWLDYGTKAYEWEKAGRSDRVQMRNDLGKINKVITEHTNYLAPYQFLTAFSQLISRICHSHDEVFVVLMEIIAKVFLAYPQQAMWMMTAVSKSSYPMRVNRCKEILNKAIHMKKSLEKFVGDATRLTDKLLELCNKPVDGSSSTLSMSTHFKMLKKLVEEATFSEILIPLQSVMIPTLPSILGTHANHASHEPFPGHWAYIAGFDDMVEILASLQKPKKISLKGSDGKFYIMMCKPKDDLRKDCRLMEFNSLINKCLRKDAESRRRELHIRTYAVIPLNDECGIIEWVNNTAGLRPILTKLYKEKGVYMTGKELRQCMLPKSAALSEKLKVFREFLLPRHPPIFHEWFLRTFPDPTSWYSSRSAYCRSTAVMSMVGYILGLGDRHGENILFDSLTGECVHVDFNCLFNKGETFEVPEIVPFRLTHNMVNGMGPMGTEGLFRRACEVTMRLMRDQREPLMSVLKTFLHDPLVEWSKPVKGHSKAPLNETGEVVNEKAKTHVLDIEQRLQGVIKTRNRVTGLPLSIEGHVHYLIQEATDENLLCQMYLGWTPYM",
# AURKC_HUMAN
"MSSPRAVVQLGKAQPAGEELATANQTAQQPSSPAMRRLTVDDFEIGRPLGKGKFGNVYLARLKESHFIVALKVLFKSQIEKEGLEHQLRREIEIQAHLQHPNILRLYNYFHDARRVYLILEYAPRGELYKELQKSEKLDEQRTATIIEELADALTYCHDKKVIHRDIKPENLLLGFRGEVKIADFGWSVHTPSLRRKTMCGTLDYLPPEMIEGRTYDEKVDLWCIGVLCYELLVGYPPFESASHSETYRRILKVDVRFPLSMPLGARDLISRLLRYQPLERLPLAQILKHPWVQAHSRRVLPPCAQMAS",
# UFO_HUMAN
"MAWRCPRMGRVPLAWCLALCGWACMAPRGTQAEESPFVGNPGNITGARGLTGTLRCQLQVQGEPPEVHWLRDGQILELADSTQTQVPLGEDEQDDWIVVSQLRITSLQLSDTGQYQCLVFLGHQTFVSQPGYVGLEGLPYFLEEPEDRTVAANTPFNLSCQAQGPPEPVDLLWLQDAVPLATAPGHGPQRSLHVPGLNKTSSFSCEAHNAKGVTTSRTATITVLPQQPRNLHLVSRQPTELEVAWTPGLSGIYPLTHCTLQAVLSDDGMGIQAGEPDPPEEPLTSQASVPPHQLRLGSLHPHTPYHIRVACTSSQGPSSWTHWLPVETPEGVPLGPPENISATRNGSQAFVHWQEPRAPLQGTLLGYRLAYQGQDTPEVLMDIGLRQEVTLELQGDGSVSNLTVCVAAYTAAGDGPWSLPVPLEAWRPGQAQPVHQLVKEPSTPAFSWPWWYVLLGAVVAAACVLILALFLVHRRKKETRYGEVFEPTVERGELVVRYRVRKSYSRRTTEATLNSLGISEELKEKLRDVMVDRHKVALGKTLGEGEFGAVMEGQLNQDDSILKVAVKTMKIAICTRSELEDFLSEAVCMKEFDHPNVMRLIGVCFQGSERESFPAPVVILPFMKHGDLHSFLLYSRLGDQPVYLPTQMLVKFMADIASGMEYLSTKRFIHRDLAARNCMLNENMSVCVADFGLSKKIYNGDYYRQGRIAKMPVKWIAIESLADRVYTSKSDVWSFGVTMWEIATRGQTPYPGVENSEIYDYLRQGNRLKQPADCLDGLYALMSRCWELNPQDRPSFTELREDLENTLKALPPAQEPDEILYVNMDEGGGYPEPPGAAGGADPPTQPDPKDSCSCLTAAEVHPAGRYVLCPSTTPSPAQPADRGSPAAPGQEDGA",
# ARBK1_HUMAN
"MADLEAVLADVSYLMAMEKSKATPAARASKKILLPEPSIRSVMQKYLEDRGEVTFEKIFSQKLGYLLFRDFCLNHLEEARPLVEFYEEIKKYEKLETEEERVARSREIFDSYIMKELLACSHPFSKSATEHVQGHLGKKQVPPDLFQPYIEEICQNLRGDVFQKFIESDKFTRFCQWKNVELNIHLTMNDFSVHRIIGRGGFGEVYGCRKADTGKMYAMKCLDKKRIKMKQGETLALNERIMLSLVSTGDCPFIVCMSYAFHTPDKLSFILDLMNGGDLHYHLSQHGVFSEADMRFYAAEIILGLEHMHNRFVVYRDLKPANILLDEHGHVRISDLGLACDFSKKKPHASVGTHGYMAPEVLQKGVAYDSSADWFSLGCMLFKLLRGHSPFRQHKTKDKHEIDRMTLTMAVELPDSFSPELRSLLEGLLQRDVNRRLGCLGRGAQEVKESPFFRSLDWQMVFLQKYPPPLIPPRGEVNAADAFDIGSFDEEDTKGIKLLDSDQELYRNFPLTISERWQQEVAETVFDTINAETDRLEARKKAKNKQLGHEEDYALGKDCIMHGYMSKMGNPFLTQWQRRYFYLFPNRLEWRGEGEAPQSLLTMEEIQSVEETQIKERKCLLLKIRGGKQFILQCDSDPELVQWKKELRDAYREAQQLVQRVPKMKNKPRSPVVELSKVPLVQRGSANGL",
# BCKD_HUMAN
"MILASVLRSGPGGGLPLRPLLGPALALRARSTSATDTHHVEMARERSKTVTSFYNQSAIDAAAEKPSVRLTPTMMLYAGRSQDGSHLLKSARYLQQELPVRIAHRIKGFRCLPFIIGCNPTILHVHELYIRAFQKLTDFPPIKDQADEAQYCQLVRQLLDDHKDVVTLLAEGLRESRKHIEDEKLVRYFLDKTLTSRLGIRMLATHHLALHEDKPDFVGIICTRLSPKKIIEKWVDFARRLCEHKYGNAPRVRINGHVAARFPFIPMPLDYILPELLKNAMRATMESHLDTPYNVPDVVITIANNDVDLIIRISDRGGGIAHKDLDRVMDYHFTTAEASTQDPRISPLFGHLDMHSGAQSGPMHGFGFGLPTSRAYAEYLGGSLQLQSLQGIGTDVYLRLRHIDGREESFRI",
# M4K2_HUMAN
"MALLRDVSLQDPRDRFELLQRVGAGTYGDVYKARDTVTSELAAVKIVKLDPGDDISSLQQEITILRECRHPNVVAYIGSYLRNDRLWICMEFCGGGSLQEIYHATGPLEERQIAYVCREALKGLHHLHSQGKIHRDIKGANLLLTLQGDVKLADFGVSGELTASVAKRRSFIGTPYWMAPEVAAVERKGGYNELCDVWALGITAIELGELQPPLFHLHPMRALMLMSKSSFQPPKLRDKTRWTQNFHHFLKLALTKNPKKRPTAEKLLQHPFTTQQLPRALLTQLLDKASDPHLGTPSPEDCELETYDMFPDTIHSRGQHGPAERTPSEIQFHQVKFGAPRRKETDPLNEPWEEEWTLLGKEELSGSLLQSVQEALEERSLTIRSASEFQELDSPDDTMGTIKRAPFLGPLPTDPPAEEPLSSPPGTLPPPPSGPNSSPLLPTAWATMKQREDPERSSCHGLPPTPKVHMGACFSKVFNGCPLRIHAAVTWIHPVTRDQFLVVGAEEGIYTLNLHELHEDTLEKLISHRCSWLYCVNNVLLSLSGKSTHIWAHDLPGLFEQRRLQQQVPLSIPTNRLTQRIIPRRFALSTKIPDTKGCLQCRVVRNPYTGATFLLAALPTSLLLLQWYEPLQKFLLLKNFSSPLPSPAGMLEPLVLDGKELPQVCVGAEGPEGPGCRVLFHVLPLEAGLTPDILIPPEGIPGSAQQVIQVDRDTILVSFERCVRIVNMQGEPTATLAPELTFDFPIETVVCLQDSVLAFWSHGMQGRSLDTNEVTQEITDETRIFRVLGAHRDIILESIPTDNPEAHSNLYILTGHQSTY",
# BLK_HUMAN
"MGLVSSKKPDKEKPIKEKDKGQWSPLKVSAQDKDAPPLPPLVVFNHLTPPPPDEHLDEDKHFVVALYDYTAMNDRDLQMLKGEKLQVLKGTGDWWLARSLVTGREGYVPSNFVARVESLEMERWFFRSQGRKEAERQLLAPINKAGSFLIRESETNKGAFSLSVKDVTTQGELIKHYKIRCLDEGGYYISPRITFPSLQALVQHYSKKGDGLCQRLTLPCVRPAPQNPWAQDEWEIPRQSLRLVRKLGSGQFGEVWMGYYKNNMKVAIKTLKEGTMSPEAFLGEANVMKALQHERLVRLYAVVTKEPIYIVTEYMARGCLLDFLKTDEGSRLSLPRLIDMSAQIAEGMAYIERMNSIHRDLRAANILVSEALCCKIADFGLARIIDSEYTAQEGAKFPIKWTAPEAIHFGVFTIKADVWSFGVLLMEVVTYGRVPYPGMSNPEVIRNLERGYRMPRPDTCPPELYRGVIAECWRSRPEERPTFEFLQSVLEDFYTATERQYELQP",
# BRAF_HUMAN
"MAALSGGGGGGAEPGQALFNGDMEPEAGAGAGAAASSAADPAIPEEVWNIKQMIKLTQEHIEALLDKFGGEHNPPSIYLEAYEEYTSKLDALQQREQQLLESLGNGTDFSVSSSASMDTVTSSSSSSLSVLPSSLSVFQNPTDVARSNPKSPQKPIVRVFLPNKQRTVVPARCGVTVRDSLKKALMMRGLIPECCAVYRIQDGEKKPIGWDTDISWLTGEELHVEVLENVPLTTHNFVRKTFFTLAFCDFCRKLLFQGFRCQTCGYKFHQRCSTEVPLMCVNYDQLDLLFVSKFFEHHPIPQEEASLAETALTSGSSPSAPASDSIGPQILTSPSPSKSIPIPQPFRPADEDHRNQFGQRDRSSSAPNVHINTIEPVNIDDLIRDQGFRGDGGSTTGLSATPPASLPGSLTNVKALQKSPGPQRERKSSSSSEDRNRMKTLGRRDSSDDWEIPDGQITVGQRIGSGSFGTVYKGKWHGDVAVKMLNVTAPTPQQLQAFKNEVGVLRKTRHVNILLFMGYSTKPQLAIVTQWCEGSSLYHHLHIIETKFEMIKLIDIARQTAQGMDYLHAKSIIHRDLKSNNIFLHEDLTVKIGDFGLATVKSRWSGSHQFEQLSGSILWMAPEVIRMQDKNPYSFQSDVYAFGIVLYELMTGQLPYSNINNRDQIIFMVGRGYLSPDLSKVRSNCPKAMKRLMAECLKKKRDERPLFPQILASIELLARSLPKIHRSASEPSLNRAGFQTEDFSLYACASPKTPIQAGGYGAFPVH",
# PTK6_HUMAN
"MVSRDQAHLGPKYVGLWDFKSRTDEELSFRAGDVFHVARKEEQWWWATLLDEAGGAVAQGYVPHNYLAERETVESEPWFFGCISRSEAVRRLQAEGNATGAFLIRVSEKPSADYVLSVRDTQAVRHYKIWRRAGGRLHLNEAVSFLSLPELVNYHRAQSLSHGLRLAAPCRKHEPEPLPHWDDWERPREEFTLCRKLGSGYFGEVFEGLWKDRVQVAIKVISRDNLLHQQMLQSEIQAMKKLRHKHILALYAVVSVGDPVYIITELMAKGSLLELLRDSDEKVLPVSELLDIAWQVAEGMCYLESQNYIHRDLAARNILVGENTLCKVGDFGLARLIKEDVYLSHDHNIPYKWTAPEALSRGHYSTKSDVWSFGILLHEMFSRGQVPYPGMSNHEAFLRVDAGYRMPCPLECPPSVHKLMLTCWCRDPEQRPCFKALRERLSSFTSYENPT",
# BTK_HUMAN
"MAAVILESIFLKRSQQKKKTSPLNFKKRLFLLTVHKLSYYEYDFERGRRGSKKGSIDVEKITCVETVVPEKNPPPERQIPRRGEESSEMEQISIIERFPYPFQVVYDEGPLYVFSPTEELRKRWIHQLKNVIRYNSDLVQKYHPCFWIDGQYLCCSQTAKNAMGCQILENRNGSLKPGSSHRKTKKPLPPTPEEDQILKKPLPPEPAAAPVSTSELKKVVALYDYMPMNANDLQLRKGDEYFILEESNLPWWRARDKNGQEGYIPSNYVTEAEDSIEMYEWYSKHMTRSQAEQLLKQEGKEGGFIVRDSSKAGKYTVSVFAKSTGDPQGVIRHYVVCSTPQSQYYLAEKHLFSTIPELINYHQHNSAGLISRLKYPVSQQNKNAPSTAGLGYGSWEIDPKDLTFLKELGTGQFGVVKYGKWRGQYDVAIKMIKEGSMSEDEFIEEAKVMMNLSHEKLVQLYGVCTKQRPIFIITEYMANGCLLNYLREMRHRFQTQQLLEMCKDVCEAMEYLESKQFLHRDLAARNCLVNDQGVVKVSDFGLSRYVLDDEYTSSVGSKFPVRWSPPEVLMYSKFSSKSDIWAFGVLMWEIYSLGKMPYERFTNSETAEHIAQGLRLYRPHLASEKVYTIMYSCWHEKADERPTFKILLSNILDVMDEES",
# BUB1B_HUMAN
"MAAVKKEGGALSEAMSLEGDEWELSKENVQPLRQGRIMSTLQGALAQESACNNTLQQQKRAFEYEIRFYTGNDPLDVWDRYISWTEQNYPQGGKESNMSTLLERAVEALQGEKRYYSDPRFLNLWLKLGRLCNEPLDMYSYLHNQGIGVSLAQFYISWAEEYEARENFRKADAIFQEGIQQKAEPLERLQSQHRQFQARVSRQTLLALEKEEEEEVFESSVPQRSTLAELKSKGKKTARAPIIRVGGALKAPSQNRGLQNPFPQQMQNNSRITVFDENADEASTAELSKPTVQPWIAPPMPRAKENELQAGPWNTGRSLEHRPRGNTASLIAVPAVLPSFTPYVEETARQPVMTPCKIEPSINHILSTRKPGKEEGDPLQRVQSHQQASEEKKEKMMYCKEKIYAGVGEFSFEEIRAEVFRKKLKEQREAELLTSAEKRAEMQKQIEEMEKKLKEIQTTQQERTGDQQEETMPTKETTKLQIASESQKIPGMTLSSSVCQVNCCARETSLAENIWQEQPHSKGPSVPFSIFDEFLLSEKKNKSPPADPPRVLAQRRPLAVLKTSESITSNEDVSPDVCDEFTGIEPLSEDAIITGFRNVTICPNPEDTCDFARAARFVSTPFHEIMSLKDLPSDPERLLPEEDLDVKTSEDQQTACGTIYSQTLSIKKLSPIIEDSREATHSSGFSGSSASVASTSSIKCLQIPEKLELTNETSENPTQSPWCSQYRRQLLKSLPELSASAELCIEDRPMPKLEIEKEIELGNEDYCIKREYLICEDYKLFWVAPRNSAELTVIKVSSQPVPWDFYINLKLKERLNEDFDHFCSCYQYQDGCIVWHQYINCFTLQDLLQHSEYITHEITVLIIYNLLTIVEMLHKAEIVHGDLSPRCLILRNRIHDPYDCNKNNQALKIVDFSYSVDLRVQLDVFTLSGFRTVQILEGQKILANCSSPYQVDLFGIADLAHLLLFKEHLQVFWDGSFWKLSQNISELKDGELWNKFFVRILNANDEATVSVLGELAAEMNGVFDTTFQSHLNKALWKVGKLTSPGALLFQ",
# CDK7_HUMAN
"MALDVKSRAKRYEKLDFLGEGQFATVYKARDKNTNQIVAIKKIKLGHRSEAKDGINRTALREIKLLQELSHPNIIGLLDAFGHKSNISLVFDFMETDLEVIIKDNSLVLTPSHIKAYMLMTLQGLEYLHQHWILHRDLKPNNLLLDENGVLKLADFGLAKSFGSPNRAYTHQVVTRWYRAPELLFGARMYGVGVDMWAVGCILAELLLRVPFLPGDSDLDQLTRIFETLGTPTEEQWPDMCSLPDYVTFKSFPGIPLHHIFSAAGDDLLDLIQGLFLFNPCARITATQALKMKYFSNRPGPTPGCQLPRPNCPVETLKEQSNPALAIKRKRTEALEQGGLPKKLIF",
# KCC1A_HUMAN
"MLGAVEGPRWKQAEDIRDIYDFRDVLGTGAFSEVILAEDKRTQKLVAIKCIAKEALEGKEGSMENEIAVLHKIKHPNIVALDDIYESGGHLYLIMQLVSGGELFDRIVEKGFYTERDASRLIFQVLDAVKYLHDLGIVHRDLKPENLLYYSLDEDSKIMISDFGLSKMEDPGSVLSTACGTPGYVAPEVLAQKPYSKAVDCWSIGVIAYILLCGYPPFYDENDAKLFEQILKAEYEFDSPYWDDISDSAKDFIRHLMEKDPEKRFTCEQALQHPWIAGDTALDKNIHQSVSEQIKKNFAKSKWKQAFNATAVVRHMRKLQLGTSQEGQGQTASHGELLTPVAGGPAAGCCCRDCCVEPGTELSPTLPHQL",
# KCC2A_HUMAN
"MATITCTRFTEEYQLFEELGKGAFSVVRRCVKVLAGQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGHHYLIFDLVTGGELFEDIVAREYYSEADASHCIQQILEAVLHCHQMGVVHRDLKPENLLLASKLKGAAVKLADFGLAIEVEGEQQAWFGFAGTPGYLSPEVLRKDPYGKPVDLWACGVILYILLVGYPPFWDEDQHRLYQQIKAGAYDFPSPEWDTVTPEAKDLINKMLTINPSKRITAAEALKHPWISHRSTVASCMHRQETVDCLKKFNARRKLKGAILTTMLATRNFSGGKSGGNKKSDGVKESSESTNTTIEDEDTKVRKQEIIKVTEQLIEAISNGDFESYTKMCDPGMTAFEPEALGNLVEGLDFHRFYFENLWSRNSKPVHTTILNPHIHLMGDESACIAYIRITQYLDAGGIPRTAQSEETRVWHRRDGKWQIVHFHRSGAPSVLPH",
# KCC2B_HUMAN
"MATTVTCTRFTDEYQLYEDIGKGAFSVVRRCVKLCTGHEYAAKIINTKKLSARDHQKLEREARICRLLKHSNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIQQILEAVLHCHQMGVVHRDLKPENLLLASKCKGAAVKLADFGLAIEVQGDQQAWFGFAGTPGYLSPEVLRKEAYGKPVDIWACGVILYILLVGYPPFWDEDQHKLYQQIKAGAYDFPSPEWDTVTPEAKNLINQMLTINPAKRITAHEALKHPWVCQRSTVASMMHRQETVECLKKFNARRKLKGAILTTMLATRNFSVGRQTTAPATMSTAASGTTMGLVEQAKSLLNKKADGVKPQTNSTKNSAAATSPKGTLPPAALEPQTTVIHNPVDGIKESSDSANTTIEDEDAKAPRVPDILSSVRRGSGAPEAEGPLPCPSPAPFSPLPAPSPRISDILNSVRRGSGTPEAEGPLSAGPPPCLSPALLGPLSSPSPRISDILNSVRRGSGTPEAEGPSPVGPPPCPSPTIPGPLPTPSRKQEIIKTTEQLIEAVNNGDFEAYAKICDPGLTSFEPEALGNLVEGMDFHRFYFENLLAKNSKPIHTTILNPHVHVIGEDAACIAYIRLTQYIDGQGRPRTSQSEETRVWHRRDGKWQNVHFHCSGAPVAPLQ",
# KCC2G_HUMAN
"MATTATCTRFTDDYQLFEELGKGAFSVVRRCVKKTSTQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIHQILESVNHIHQHDIVHRDLKPENLLLASKCKGAAVKLADFGLAIEVQGEQQAWFGFAGTPGYLSPEVLRKDPYGKPVDIWACGVILYILLVGYPPFWDEDQHKLYQQIKAGAYDFPSPEWDTVTPEAKNLINQMLTINPAKRITADQALKHPWVCQRSTVASMMHRQETVECLRKFNARRKLKGAILTTMLVSRNFSAAKSLLNKKSDGGVKKRKSSSSVHLMPQSNNKNSLVSPAQEPAPLQTAMEPQTTVVHNATDGIKGSTESCNTTTEDEDLKGRVPEGRSSRDRTAPSAGMQPQPSLCSSAMRKQEIIKITEQLIEAINNGDFEAYTKICDPGLTSFEPEALGNLVEGMDFHKFYFENLLSKNSKPIHTTILNPHVHVIGEDAACIAYIRLTQYIDGQGRPRTSQSEETRVWHRRDGKWLNVHYHCSGAPAAPLQ",
# KCC4_HUMAN
"MLKVTVPSCSASSCSSVTASAAPGTASLVPDYWIDGSNRDALSDFFEVESELGRGATSIVYRCKQKGTQKPYALKVLKKTVDKKIVRTEIGVLLRLSHPNIIKLKEIFETPTEISLVLELVTGGELFDRIVEKGYYSERDAADAVKQILEAVAYLHENGIVHRDLKPENLLYATPAPDAPLKIADFGLSKIVEHQVLMKTVCGTPGYCAPEILRGCAYGPEVDMWSVGIITYILLCGFEPFYDERGDQFMFRRILNCEYYFISPWWDEVSLNAKDLVRKLIVLDPKKRLTTFQALQHPWVTGKAANFVHMDTAQKKLQEFNARRKLKAAVKAVVASSRLGSASSSHGSIQESHKASRDPSPIQDGNEDMKAIPEGEKIQGDGAQAAVKGAQAELMKVQALEKVKGADINAEEAPKMVPKAVEDGIKVADLELEEGLAEEKLKTVEEAAAPREGQGSSAVGFEVPQQDVILPEY",
# CAMKV_HUMAN
"MPFGCVTLGDKKNYNQPSEVTDRYDLGQVIKTEEFCEIFRAKDKTTGKLHTCKKFQKRDGRKVRKAAKNEIGILKMVKHPNILQLVDVFVTRKEYFIFLELATGREVFDWILDQGYYSERDTSNVVRQVLEAVAYLHSLKIVHRNLKLENLVYYNRLKNSKIVISDFHLAKLENGLIKEPCGTPEYLAPEVVGRQRYGRPVDCWAIGVIMYILLSGNPPFYEEVEEDDYENHDKNLFRKILAGDYEFDSPYWDDISQAAKDLVTRLMEVEQDQRITAEEAISHEWISGNAASDKNIKDGVCAQIEKNFARAKWKKAVRVTTLMKRLRAPEQSSTAAAQSASATDTATPGAAGGATAAAASGATSAPEGDAARAAKSDNVAPADRSATPATDGSATPATDGSVTPATDGSITPATDGSVTPATDRSATPATDGRATPATEESTVPTTQSSAMLATKAAATPEPAMAQPDSTAPEGATGQAPPSSKGEEAAGYAQESQREEAS",
# DCLK1_HUMAN
"MSFGRDMELEHFDERDKAQRYSRGSRVNGLPSPTHSAHCSFYRTRTLQTLSSEKKAKKVRFYRNGDRYFKGIVYAISPDRFRSFEALLADLTRTLSDNVNLPQGVRTIYTIDGLKKISSLDQLVEGESYVCGSIEPFKKLEYTKNVNPNWSVNVKTTSASRAVSSLATAKGSPSEVRENKDFIRPKLVTIIRSGVKPRKAVRILLNKKTAHSFEQVLTDITDAIKLDSGVVKRLYTLDGKQVMCLQDFFGDDDIFIACGPEKFRYQDDFLLDESECRVVKSTSYTKIASSSRRSTTKSPGPSRRSKSPASTSSVNGTPGSQLSTPRSGKSPSPSPTSPGSLRKQRSSQHGGSSTSLASTKVCSSMDENDGPGEEVSEEGFQIPATITERYKVGRTIGDGNFAVVKECVERSTAREYALKIIKKSKCRGKEHMIQNEVSILRRVKHPNIVLLIEEMDVPTELYLVMELVKGGDLFDAITSTNKYTERDASGMLYNLASAIKYLHSLNIVHRDIKPENLLVYEHQDGSKSLKLGDFGLATIVDGPLYTVCGTPTYVAPEIIAETGYGLKVDIWAAGVITYILLCGFPPFRGSGDDQEVLFDQILMGQVDFPSPYWDNVSDSAKELITMMLLVDVDQRFSAVQVLEHPWVNDDGLPENEHQLSVAGKIKKHFNTGPKPNSTAAGVSVIATTALDKERQVFRRRRNQDVRSRYKAQPAPPELNSESEDYSPSSSETVRSPNSPF",
# CSKP_HUMAN
"MADDDVLFEDVYELCEVIGKGPFSVVRRCINRETGQQFAVKIVDVAKFTSSPGLSTEDLKREASICHMLKHPHIVELLETYSSDGMLYMVFEFMDGADLCFEIVKRADAGFVYSEAVASHYMRQILEALRYCHDNNIIHRDVKPHCVLLASKENSAPVKLGGFGVAIQLGESGLVAGGRVGTPHFMAPEVVKREPYGKPVDVWGCGVILFILLSGCLPFYGTKERLFEGIIKGKYKMNPRQWSHISESAKDLVRRMLMLDPAERITVYEALNHPWLKERDRYAYKIHLPETVEQLRKFNARRKLKGAVLAAVSSHKFNSFYGDPPEELPDFSEDPTSSGLLAAERAVSQVLDSLEEIHALTDCSEKDLDFLHSVFQDQHLHTLLDLYDKINTKSSPQIRNPPSDAVQRAKEVLEEISCYPENNDAKELKRILTQPHFMALLQTHDVVAHEVYSDEALRVTPPPTSPYLNGDSPESANGDMDMENVTRVRLVQFQKNTDEPMGITLKMNELNHCIVARIMHGGMIHRQGTLHVGDEIREINGISVANQTVEQLQKMLREMRGSITFKIVPSYRTQSSSCERDSPSTSRQSPANGHSSTNNSVSDLPSTTQPKGRQIYVRAQFEYDPAKDDLIPCKEAGIRFRVGDIIQIISKDDHNWWQGKLENSKNGTAGLIPSPELQEWRVACIAMEKTKQEQQASCTWFGKKKKQYKDKYLAKHNAVFDQLDLVTYEEVVKLPAFKRKTLVLLGAHGVGRRHIKNTLITKHPDRFAYPIPHTTRPPKKDEENGKNYYFVSHDQMMQDISNNEYLEYGSHEDAMYGTKLETIRKIHEQGLIAILDVEPQALKVLRTAEFAPFVVFIAAPTITPGLNEDESLQRLQKESDILQRTYAHYFDLTIINNEIDETIRHLEEAVELVCTAPQWVPVSWVY",
# CDK1_HUMAN
"MEDYTKIEKIGEGTYGVVYKGRHKTTGQVVAMKKIRLESEEEGVPSTAIREISLLKELRHPNIVSLQDVLMQDSRLYLIFEFLSMDLKKYLDSIPPGQYMDSSLVKSYLYQILQGIVFCHSRRVLHRDLKPQNLLIDDKGTIKLADFGLARAFGIPIRVYTHEVVTLWYRSPEVLLGSARYSTPVDIWSIGTIFAELATKKPLFHGDSEIDQLFRIFRALGTPNNEVWPEVESLQDYKNTFPKWKPGSLASHVKNLDENGLDLLSKMLIYDPAKRISGKMALNHPYFNDLDNQIKKM",
# CDC7_HUMAN
"MEASLGIQMDEPMAFSPQRDRFQAEGSLKKNEQNFKLAGVKKDIEKLYEAVPQLSNVFKIEDKIGEGTFSSVYLATAQLQVGPEEKIALKHLIPTSHPIRIAAELQCLTVAGGQDNVMGVKYCFRKNDHVVIAMPYLEHESFLDILNSLSFQEVREYMLNLFKALKRIHQFGIVHRDVKPSNFLYNRRLKKYALVDFGLAQGTHDTKIELLKFVQSEAQQERCSQNKSHIITGNKIPLSGPVPKELDQQSTTKASVKRPYTNAQIQIKQGKDGKEGSVGLSVQRSVFGERNFNIHSSISHESPAVKLMKQSKTVDVLSRKLATKKKAISTKVMNSAVMRKTASSCPASLTCDCYATDKVCSICLSRRQQVAPRAGTPGFRAPEVLTKCPNQTTAIDMWSAGVIFLSLLSGRYPFYKASDDLTALAQIMTIRGSRETIQAAKTFGKSILCSKEVPAQDLRKLCERLRGMDSSTPKLTSDIQGHASHQPAISEKTDHKASCLVQTPPGQYSGNSFKKGDSNSCEHCFDEYNTNLEGWNEVPDEAYDLLDKLLDLNPASRITAEEALLHPFFKDMSL",
# CDK2_HUMAN
"MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNHPNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHSHRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYYSTAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSFPKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL",
# CDK3_HUMAN
"MDMFQKVEKIGEGTYGVVYKAKNRETGQLVALKKIRLDLEMEGVPSTAIREISLLKELKHPNIVRLLDVVHNERKLYLVFEFLSQDLKKYMDSTPGSELPLHLIKSYLFQLLQGVSFCHSHRVIHRDLKPQNLLINELGAIKLADFGLARAFGVPLRTYTHEVVTLWYRAPEILLGSKFYTTAVDIWSIGCIFAEMVTRKALFPGDSEIDQLFRIFRMLGTPSEDTWPGVTQLPDYKGSFPKWTRKGLEEIVPNLEPEGRDLLMQLLQYDPSQRITAKTALAHPYFSSPEPSPAARQYVLQRFRH",
# CDK4_HUMAN
"MATSRYEPVAEIGVGAYGTVYKARDPHSGHFVALKSVRVPNGGGGGGGLPISTVREVALLRRLEAFEHPNVVRLMDVCATSRTDREIKVTLVFEHVDQDLRTYLDKAPPPGLPAETIKDLMRQFLRGLDFLHANCIVHRDLKPENILVTSGGTVKLADFGLARIYSYQMALTPVVVTLWYRAPEVLLQSTYATPVDMWSVGCIFAEMFRRKPLFCGNSEADQLGKIFDLIGLPPEDDWPRDVSLPRGAFPPRGPRPVQSVVPEMEESGAQLLLEMLTFNPHKRISAFRALQHSYLHKDEGNPE",
# CDK5_HUMAN
"MQKYEKLEKIGEGTYGTVFKAKNRETHEIVALKRVRLDDDDEGVPSSALREICLLKELKHKNIVRLHDVLHSDKKLTLVFEFCDQDLKKYFDSCNGDLDPEIVKSFLFQLLKGLGFCHSRNVLHRDLKPQNLLINRNGELKLADFGLARAFGIPVRCYSAEVVTLWYRPPDVLFGAKLYSTSIDMWSAGCIFAELANAGRPLFPGNDVDDQLKRIFRLLGTPTEEQWPSMTKLPDYKPYPMYPATTSLVNVVPKLNATGRDLLQNLLKCNPVQRISAEEALQHPYFSDFCPP",
# CDK6_HUMAN
"MEKDGLCRADQQYECVAEIGEGAYGKVFKARDLKNGGRFVALKRVRVQTGEEGMPLSTIREVAVLRHLETFEHPNVVRLFDVCTVSRTDRETKLTLVFEHVDQDLTTYLDKVPEPGVPTETIKDMMFQLLRGLDFLHSHRVVHRDLKPQNILVTSSGQIKLADFGLARIYSFQMALTSVVVTLWYRAPEVLLQSSYATPVDLWSVGCIFAEMFRRKPLFRGSSDVDQLGKILDVIGLPGEEDWPRDVALPRQAFHSKSAQPIEKFVTDIDELGKDLLLKCLTFNPAKRISAYSALSHPYFQDLERCKENLDSHLPPSQNTSELNTA",
# CDK8_HUMAN
"MDYDFKVKLSSERERVEDLFEYEGCKVGRGTYGHVYKAKRKDGKDDKDYALKQIEGTGISMSACREIALLRELKHPNVISLQKVFLSHADRKVWLLFDYAEHDLWHIIKFHRASKANKKPVQLPRGMVKSLLYQILDGIHYLHANWVLHRDLKPANILVMGEGPERGRVKIADMGFARLFNSPLKPLADLDPVVVTFWYRAPELLLGARHYTKAIDIWAIGCIFAELLTSEPIFHCRQEDIKTSNPYHHDQLDRIFNVMGFPADKDWEDIKKMPEHSTLMKDFRRNTYTNCSLIKYMEKHKVKPDSKAFHLLQKLLTMDPIKRITSEQAMQDPYFLEDPLPTSDVFAGCQIPYPKREFLTEEEPDDKGDKKNQQQQQGNNHTNGTGHPGNQDSSHTQGPPLKKVRVVPPTTTSGGLIMTSDYQRSNPHAAYPNPGPSTSQPQSSMGYSATSQQPPQYSHQTHRY",
# KGP1_HUMAN
"MSELEEDFAKILMLKEERIKELEKRLSEKEEEIQELKRKLHKCQSVLPVPSTHIGPRTTRAQGISAEPQTYRSFHDLRQAFRKFTKSERSKDLIKEAILDNDFMKNLELSQIQEIVDCMYPVEYGKDSCIIKEGDVGSLVYVMEDGKVEVTKEGVKLCTMGPGKVFGELAILYNCTRTATVKTLVNVKLWAIDRQCFQTIMMRTGLIKHTEYMEFLKSVPTFQSLPEEILSKLADVLEETHYENGEYIIRQGARGDTFFIISKGTVNVTREDSPSEDPVFLRTLGKGDWFGEKALQGEDVRTANVIAAEAVTCLVIDRDSFKHLIGGLDDVSNKAYEDAEAKAKYEAEAAFFANLKLSDFNIIDTLGVGGFGRVELVQLKSEESKTFAMKILKKRHIVDTRQQEHIRSEKQIMQGAHSDFIVRLYRTFKDSKYLYMLMEACLGGELWTILRDRGSFEDSTTRFYTACVVEAFAYLHSKGIIYRDLKPENLILDHRGYAKLVDFGFAKKIGFGKKTWTFCGTPEYVAPEIILNKGHDISADYWSLGILMYELLTGSPPFSGPDPMKTYNIILRGIDMIEFPKKIAKNAANLIKKLCRDNPSERLGNLKNGVKDIQKHKWFEGFNWEGLRKGTLTPPIIPSVASPTDTSNFDSFPEDNDEPPPDDNSGWDIDF",
# KGP2_HUMAN
"MGNGSVKPKHSKHPDGHSGNLTTDALRNKVTELERELRRKDAEIQEREYHLKELREQLSKQTVAIAELTEELQNKCIQLNKLQDVVHMQGGSPLQASPDKVPLEVHRKTSGLVSLHSRRGAKAGVSAEPTTRTYDLNKPPEFSFEKARVRKDSSEKKLITDALNKNQFLKRLDPQQIKDMVECMYGRNYQQGSYIIKQGEPGNHIFVLAEGRLEVFQGEKLLSSIPMWTTFGELAILYNCTRTASVKAITNVKTWALDREVFQNIMRRTAQARDEQYRNFLRSVSLLKNLPEDKLTKIIDCLEVEYYDKGDYIIREGEEGSTFFILAKGKVKVTQSTEGHDQPQLIKTLQKGEYFGEKALISDDVRSANIIAEENDVACLVIDRETFNQTVGTFEELQKYLEGYVANLNRDDEKRHAKRSMSNWKLSKALSLEMIQLKEKVARFSSSSPFQNLEIIATLGVGGFGRVELVKVKNENVAFAMKCIRKKHIVDTKQQEHVYSEKRILEELCSPFIVKLYRTFKDNKYVYMLLEACLGGELWSILRDRGSFDEPTSKFCVACVTEAFDYLHRLGIIYRDLKPENLILDAEGYLKLVDFGFAKKIGSGQKTWTFCGTPEYVAPEVILNKGHDFSVDFWSLGILVYELLTGNPPFSGVDQMMTYNLILKGIEKMDFPRKITRRPEDLIRRLCRQNPTERLGNLKNGINDIKKHRWLNGFNWEGLKARSLPSPLQRELKGPIDHSYFDKYPPEKGMPPDELSGWDKDF",
# CDK13_HUMAN
"MPSSSDTALGGGGGLSWAEKKLEERRKRRRFLSPQQPPLLLPLLQPQLLQPPPPPPPLLFLAAPGTAAAAAAAAAASSSCFSPGPPLEVKRLARGKRRAGGRQKRRRGPRAGQEAEKRRVFSLPQPQQDGGGGASSGGGVTPLVEYEDVSSQSEQGLLLGGASAATAATAAGGTGGSGGSPASSSGTQRRGEGSERRPRRDRRSSSGRSKERHREHRRRDGQRGGSEASKSRSRHSHSGEERAEVAKSGSSSSSGGRRKSASATSSSSSSRKDRDSKAHRSRTKSSKEPPSAYKEPPKAYREDKTEPKAYRRRRSLSPLGGRDDSPVSHRASQSLRSRKSPSPAGGGSSPYSRRLPRSPSPYSRRRSPSYSRHSSYERGGDVSPSPYSSSSWRRSRSPYSPVLRRSGKSRSRSPYSSRHSRSRSRHRLSRSRSRHSSISPSTLTLKSSLAAELNKNKKARAAEAARAAEAAKAAEATKAAEAAAKAAKASNTSTPTKGNTETSASASQTNHVKDVKKIKIEHAPSPSSGGTLKNDKAKTKPPLQVTKVENNLIVDKATKKAVIVGKESKSAATKEESVSLKEKTKPLTPSIGAKEKEQHVALVTSTLPPLPLPPMLPEDKEADSLRGNISVKAVKKEVEKKLRCLLADLPLPPELPGGDDLSKSPEEKKTATQLHSKRRPKICGPRYGETKEKDIDWGKRCVDKFDIIGIIGEGTYGQVYKARDKDTGEMVALKKVRLDNEKEGFPITAIREIKILRQLTHQSIINMKEIVTDKEDALDFKKDKGAFYLVFEYMDHDLMGLLESGLVHFNENHIKSFMRQLMEGLDYCHKKNFLHRDIKCSNILLNNRGQIKLADFGLARLYSSEESRPYTNKVITLWYRPPELLLGEERYTPAIDVWSCGCILGELFTKKPIFQANQELAQLELISRICGSPCPAVWPDVIKLPYFNTMKPKKQYRRKLREEFVFIPAAALDLFDYMLALDPSKRCTAEQALQCEFLRDVEPSKMPPPDLPLWQDCHELWSKKRRRQKQMGMTDDVSTIKAPRKDLSLGLDDSRTNTPQGVLPSSQLKSQGSSNVAPVKTGPGQHLNHSELAILLNLLQSKTSVNMADFVQVLNIKVNSETQQQLNKINLPAGILATGEKQTDPSTPQQESSKPLGGIQPSSQTIQPKVETDAAQAAVQSAFAVLLTQLIKAQQSKQKDVLLEERENGSGHEASLQLRPPPEPSTPVSGQDDLIQHQDMRILELTPEPDRPRILPPDQRPPEPPEPPPVTEEDLDYRTENQHVPTTSSSLTDPHAGVKAALLQLLAQHQPQDDPKREGGIDYQAGDTYVSTSDYKDNFGSSSFSSAPYVSNDGLGSSSAPPLERRSFIGNSDIQSLDNYSTASSHSGGPPQPSAFSESFPSSVAGYGDIYLNAGPMLFSGDKDHRFEYSHGPIAVLANSSDPSTGPESTHPLPAKMHNYNYGGNLQENPSGPSLMHGQTWTSPAQGPGYSQGYRGHISTSTGRGRGRGLPY",
# CHK1_HUMAN
"MAVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAVDCPENIKKEICINKMLNHENVVKFYGHRREGNIQYLFLEYCSGGELFDRIEPDIGMPEPDAQRFFHQLMAGVVYLHGIGITHRDIKPENLLLDERDNLKISDFGLATVFRYNNRERLLNKMCGTLPYVAPELLKRREFHAEPVDVWSCGIVLTAMLAGELPWDQPSDSCQEYSDWKEKKTYLNPWKKIDSAPLALLHKILVENPSARITIPDIKKDRWYNKPLKKGAKRPRVTSGGVSESPSGFSKHIQSNLDFSPVNSASSEENVKYSSSQPEPRTGLSLWDTSPSYIDKLVQGISFSQPTCPDHMLLNSQLLGTPGSSQNPWQRLVKRMTRFFTKLDADKSYQCLKETCEKLGYQWKKSCMNQVTISTTDRRNNKLIFKVNLLEMDDKILVDFRLSKGDGLEFKRHFLKIKGKLIDIVSSQKIWLPAT",
# CHK2_HUMAN
"MSRESDVEAQQSHGSSACSQPHGSVTQSQGSSSQSQGISSSSTSTMPNSSQSSHSSSGTLSSLETVSTQELYSIPEDQEPEDQEPEEPTPAPWARLWALQDGFANLECVNDNYWFGRDKSCEYCFDEPLLKRTDKYRTYSKKHFRIFREVGPKNSYIAYIEDHSGNGTFVNTELVGKGKRRPLNNNSEIALSLSRNKVFVFFDLTVDDQSVYPKALRDEYIMSKTLGSGACGEVKLAFERKTCKKVAIKIISKRKFAIGSAREADPALNVETEIEILKKLNHPCIIKIKNFFDAEDYYIVLELMEGGELFDKVVGNKRLKEATCKLYFYQMLLAVQYLHENGIIHRDLKPENVLLSSQEEDCLIKITDFGHSKILGETSLMRTLCGTPTYLAPEVLVSVGTAGYNRAVDCWSLGVILFICLSGYPPFSEHRTQVSLKDQITSGKYNFIPEVWAEVSEKALDLVKKLLVVDPKARFTTEEALRHPWLQDEDMKRKFQDLLSEENESTALPQVLAQPSTSRKRPREGEAEGAETTKRPAVCAAVL",
# KC1A_HUMAN
"MASSSGSKAEFIVGGKYKLVRKIGSGSFGDIYLAINITNGEEVAVKLESQKARHPQLLYESKLYKILQGGVGIPHIRWYGQEKDYNVLVMDLLGPSLEDLFNFCSRRFTMKTVLMLADQMISRIEYVHTKNFIHRDIKPDNFLMGIGRHCNKLFLIDFGLAKKYRDNRTRQHIPYREDKNLTGTARYASINAHLGIEQSRRDDMESLGYVLMYFNRTSLPWQGLKAATKKQKYEKISEKKMSTPVEVLCKGFPAEFAMYLNYCRGLRFEEAPDYMYLRQLFRILFRTLNHQYDYTFDWTMLKQKAAQQAASSSGQGQQAQTPTGKQTDKTKSNMKGF",
# KC1D_HUMAN
"MELRVGNRYRLGRKIGSGSFGDIYLGTDIAAGEEVAIKLECVKTKHPQLHIESKIYKMMQGGVGIPTIRWCGAEGDYNVMVMELLGPSLEDLFNFCSRKFSLKTVLLLADQMISRIEYIHSKNFIHRDVKPDNFLMGLGKKGNLVYIIDFGLAKKYRDARTHQHIPYRENKNLTGTARYASINTHLGIEQSRRDDLESLGYVLMYFNLGSLPWQGLKAATKRQKYERISEKKMSTPIEVLCKGYPSEFATYLNFCRSLRFDDKPDYSYLRQLFRNLFHRQGFSYDYVFDWNMLKFGASRAADDAERERRDREERLRHSRNPATRGLPSTASGRLRGTQEVAPPTPLTPTSHTANTSPRPVSGMERERKVSMRLHRGAPVNISSSDLTGRQDTSRMSTSQIPGRVASSGLQSVVHR",
# KC1E_HUMAN
"MELRVGNKYRLGRKIGSGSFGDIYLGANIASGEEVAIKLECVKTKHPQLHIESKFYKMMQGGVGIPSIKWCGAEGDYNVMVMELLGPSLEDLFNFCSRKFSLKTVLLLADQMISRIEYIHSKNFIHRDVKPDNFLMGLGKKGNLVYIIDFGLAKKYRDARTHQHIPYRENKNLTGTARYASINTHLGIEQSRRDDLESLGYVLMYFNLGSLPWQGLKAATKRQKYERISEKKMSTPIEVLCKGYPSEFSTYLNFCRSLRFDDKPDYSYLRQLFRNLFHRQGFSYDYVFDWNMLKFGAARNPEDVDRERREHEREERMGQLRGSATRALPPGPPTGATANRLRSAAEPVASTPASRIQPAGNTSPRAISRVDRERKVSMRLHRGAPANVSSSDLTGRQEVSRIPASQTSVPFDHLGK",
# KC1G2_HUMAN
"MDFDKKGGKGETEEGRRMSKAGGGRSSHGIRSSGTSSGVLMVGPNFRVGKKIGCGNFGELRLGKNLYTNEYVAIKLEPIKSRAPQLHLEYRFYKQLSATEGVPQVYYFGPCGKYNAMVLELLGPSLEDLFDLCDRTFTLKTVLMIAIQLITRMEYVHTKSLIYRDVKPENFLVGRPGTKRQHAIHIIDFGLAKEYIDPETKKHIPYREHKSLTGTARYMSINTHLGKEQSRRDDLEALGHMFMYFLRGSLPWQGLKADTLKERYQKIGDTKRATPIEVLCENFPEEMATYLRYVRRLDFFEKPDYDYLRKLFTDLFDRSGFVFDYEYDWAGKPLPTPIGTVHTDLPSQPQLRDKTQPHSKNQALNSTNGELNADDPTAGHSNAPITAPAEVEVADETKCCCFFKRRKRKSLQRHK",
# KC1G3_HUMAN
"MENKKKDKDKSDDRMARPSGRSGHNTRGTGSSSSGVLMVGPNFRVGKKIGCGNFGELRLGKNLYTNEYVAIKLEPMKSRAPQLHLEYRFYKQLGSGDGIPQVYYFGPCGKYNAMVLELLGPSLEDLFDLCDRTFSLKTVLMIAIQLISRMEYVHSKNLIYRDVKPENFLIGRPGNKTQQVIHIIDFGLAKEYIDPETKKHIPYREHKSLTGTARYMSINTHLGKEQSRRDDLEALGHMFMYFLRGSLPWQGLKADTLKERYQKIGDTKRATPIEVLCENFPEMATYLRYVRRLDFFEKPDYDYLRKLFTDLFDRKGYMFDYEYDWIGKQLPTPVGAVQQDPALSSNREAHQHRDKMQQSKNQSADHRAAWDSQQANPHHLRAHLAADRHGGSVQVVSSTNGELNTDDPTAGRSNAPITAPTEVEVMDETKCCCFFKRRKRKTIQRHK",
# CSK21_HUMAN
"MSGPVPSRARVYTDVNTHRPREYWDYESHVVEWGNQDDYQLVRKLGRGKYSEVFEAINITNNEKVVVKILKPVKKKKIKREIKILENLRGGPNIITLADIVKDPVSRTPALVFEHVNNTDFKQLYQTLTDYDIRFYMYEILKALDYCHSMGIMHRDVKPHNVMIDHEHRKLRLIDWGLAEFYHPGQEYNVRVASRYFKGPELLVDYQMYDYSLDMWSLGCMLASMIFRKEPFFHGHDNYDQLVRIAKVLGTEDLYDYIDKYNIELDPRFNDILGRHSRKRWERFVHSENQHLVSPEALDFLDKLLRYDHQSRLTAREAMEHPYFYTVVKDQARMGSSSMPGGSTPVSSANMMSGISSVPTPSPLGPLAGSPVIAAANPLGMPVPAAAGAQQ",
# CSK22_HUMAN
"MPGPAAGSRARVYAEVNSLRSREYWDYEAHVPSWGNQDDYQLVRKLGRGKYSEVFEAINITNNERVVVKILKPVKKKKIKREVKILENLRGGTNIIKLIDTVKDPVSKTPALVFEYINNTDFKQLYQILTDFDIRFYMYELLKALDYCHSKGIMHRDVKPHNVMIDHQQKKLRLIDWGLAEFYHPAQEYNVRVASRYFKGPELLVDYQMYDYSLDMWSLGCMLASMIFRREPFFHGQDNYDQLVRIAKVLGTEELYGYLKKYHIDLDPHFNDILGQHSRKRWENFIHSENRHLVSPEALDLLDKLLRYDHQQRLTAKEAMEHPYFYPVVKEQSQPCADNAVLSSGLTAAR",
# CLK1_HUMAN
"MRHSKRTYCPDWDDKDWDYGKWRSSSSHKRRKRSHSSAQENKRCKYNHSKMCDSHYLESRSINEKDYHSRRYIDEYRNDYTQGCEPGHRQRDHESRYQNHSSKSSGRSGRSSYKSKHRIHHSTSHRRSHGKSHRRKRTRSVEDDEEGHLICQSGDVLSARYEIVDTLGEGAFGKVVECIDHKAGGRHVAVKIVKNVDRYCEAARSEIQVLEHLNTTDPNSTFRCVQMLEWFEHHGHICIVFELLGLSTYDFIKENGFLPFRLDHIRKMAYQICKSVNFLHSNKLTHTDLKPENILFVQSDYTEAYNPKIKRDERTLINPDIKVVDFGSATYDDEHHSTLVSTRHYRAPEVILALGWSQPCDVWSIGCILIEYYLGFTVFPTHDSKEHLAMMERILGPLPKHMIQKTRKRKYFHHDRLDWDEHSSAGRYVSRRCKPLKEFMLSQDVEHERLFDLIQKMLEYDPAKRITLREALKHPFFDLLKKSI",
# CLK2_HUMAN
"MPHPRRYHSSERGSRGSYREHYRSRKHKRRRSRSWSSSSDRTRRRRREDSYHVRSRSSYDDRSSDRRVYDRRYCGSYRRNDYSRDRGDAYYDTDYRHSYEYQRENSSYRSQRSSRRKHRRRRRRSRTFSRSSSQHSSRRAKSVEDDAEGHLIYHVGDWLQERYEIVSTLGEGTFGRVVQCVDHRRGGARVALKIIKNVEKYKEAARLEINVLEKINEKDPDNKNLCVQMFDWFDYHGHMCISFELLGLSTFDFLKDNNYLPYPIHQVRHMAFQLCQAVKFLHDNKLTHTDLKPENILFVNSDYELTYNLEKKRDERSVKSTAVRVVDFGSATFDHEHHSTIVSTRHYRAPEVILELGWSQPCDVWSIGCIIFEYYVGFTLFQTHDNREHLAMMERILGPIPSRMIRKTRKQKYFYRGRLDWDENTSAGRYVRENCKPLRRYLTSEAEEHHQLFDLIESMLEYEPAKRLTLGEALQHPFFARLRAEPPNKLWDSSRDISR",
# CLK3_HUMAN
"MHHCKRYRSPEPDPYLSYRWKRRRSYSREHEGRLRYPSRREPPPRRSRSRSHDRLPYQRRYRERRDSDTYRCEERSPSFGEDYYGPSRSRHRRRSRERGPYRTRKHAHHCHKRRTRSCSSASSRSQQSSKRSSRSVEDDKEGHLVCRIGDWLQERYEIVGNLGEGTFGKVVECLDHARGKSQVALKIIRNVGKYREAARLEINVLKKIKEKDKENKFLCVLMSDWFNFHGHMCIAFELLGKNTFEFLKENNFQPYPLPHVRHMAYQLCHALRFLHENQLTHTDLKPENILFVNSEFETLYNEHKSCEEKSVKNTSIRVADFGSATFDHEHHTTIVATRHYRPPEVILELGWAQPCDVWSIGCILFEYYRGFTLFQTHENREHLVMMEKILGPIPSHMIHRTRKQKYFYKGGLVWDENSSDGRYVKENCKPLKSYMLQDSLEHVQLFDLMRRMLEFDPAQRITLAEALLHPFFAGLTPEERSFHTSRNPSR",
# M3K8_HUMAN
"MEYMSTGSDNKEEIDLLIKHLNVSDVIDIMENLYASEEPAVYEPSLMTMCQDSNQNDERSKSLLLSGQEVPWLSSVRYGTVEDLLAFANHISNTAKHFYGQRPQESGILLNMVITPQNGRYQIDSDVLLIPWKLTYRNIGSDFIPRGAFGKVYLAQDIKTKKRMACKLIPVDQFKPSDVEIQACFRHENIAELYGAVLWGETVHLFMEAGEGGSVLEKLESCGPMREFEIIWVTKHVLKGLDFLHSKKVIHHDIKPSNIVFMSTKAVLVDFGLSVQMTEDVYFPKDLRGTEIYMSPEVILCRGHSTKADIYSLGATLIHMQTGTPPWVKRYPRSAYPSYLYIIHKQAPPLEDIADDCSPGMRELIEASLERNPNHRPRAADLLKHEALNPPREDQPRCQSLDSALLERKRLLSRKELELPENIADSSCTGSTEESEMLKRQRSLYIDLGALAGYFNLVRGPPTLEYG",
# CSF1R_HUMAN
"MGPGVLLLLLVATAWHGQGIPVIEPSVPELVVKPGATVTLRCVGNGSVEWDGPPSPHWTLYSDGSSSILSTNNATFQNTGTYRCTEPGDPLGGSAAIHLYVKDPARPWNVLAQEVVVFEDQDALLPCLLTDPVLEAGVSLVRVRGRPLMRHTNYSFSPWHGFTIHRAKFIQSQDYQCSALMGGRKVMSISIRLKVQKVIPGPPALTLVPAELVRIRGEAAQIVCSASSVDVNFDVFLQHNNTKLAIPQQSDFHNNRYQKVLTLNLDQVDFQHAGNYSCVASNVQGKHSTSMFFRVVESAYLNLSSEQNLIQEVTVGEGLNLKVMVEAYPGLQGFNWTYLGPFSDHQPEPKLANATTKDTYRHTFTLSLPRLKPSEAGRYSFLARNPGGWRALTFELTLRYPPEVSVIWTFINGSGTLLCAASGYPQPNVTWLQCSGHTDRCDEAQVLQVWDDPYPEVLSQEPFHKVTVQSLLTVETLEHNQTYECRAHNSVGSGSWAFIPISAGAHTHPPDEFLFTPVVVACMSIMALLLLLLLLLLYKYKQKPKYQVRWKIIESYEGNSYTFIDPTQLPYNEKWEFPRNNLQFGKTLGAGAFGKVVEATAFGLGKEDAVLKVAVKMLKSTAHADEKEALMSELKIMSHLGQHENIVNLLGACTHGGPVLVITEYCCYGDLLNFLRRKAEAMLGPSLSPGQDPEGGVDYKNIHLEKKYVRRDSGFSSQGVDTYVEMRPVSTSSNDSFSEQDLDKEDGRPLELRDLLHFSSQVAQGMAFLASKNCIHRDVAARNVLLTNGHVAKIGDFGLARDIMNDSNYIVKGNARLPVKWMAPESIFDCVYTVQSDVWSYGILLWEIFSLGLNPYPGILVNSKFYKLVKDGYQMAQPAFAPKNIYSIMQACWALEPTHRPTFQQICSFLQEQAQEDRRERDYTNLPSSSRSGGSGSSSSELEEESSSEHLTCCEQGDIAQPLLQPNNYQFC",
# CSK_HUMAN
"MSAIQAAWPSGTECIAKYNFHGTAEQDLPFCKGDVLTIVAVTKDPNWYKAKNKVGREGIIPANYVQKREGVKAGTKLSLMPWFHGKITREQAERLLYPPETGLFLVRESTNYPGDYTLCVSCDGKVEHYRIMYHASKLSIDEEVYFENLMQLVEHYTSDADGLCTRLIKPKVMEGTVAAQDEFYRSGWALNMKELKLLQTIGKGEFGDVMLGDYRGNKVAVKCIKNDATAQAFLAEASVMTQLRHSNLVQLLGVIVEEKGGLYIVTEYMAKGSLVDYLRSRGRSVLGGDCLLKFSLDVCEAMEYLEGNNFVHRDLAARNVLVSEDNVAKVSDFGLTKEASSTQDTGKLPVKWTAPEALREKKFSTKSDVWSFGILLWEIYSFGRVPYPRIPLKDVVPRVEKGYKMDAPDGCPPAVYEVMKNCWHLDAAMRPSFLQLREQLEHIKTHELHL",
# MARK3_HUMAN
"MSTRTPLPTVNERDTENHTSHGDGRQEVTSRTSRSGARCRNSIASCADEQPHIGNYRLLKTIGKGNFAKVKLARHILTGREVAIKIIDKTQLNPTSLQKLFREVRIMKILNHPNIVKLFEVIETEKTLYLIMEYASGGEVFDYLVAHGRMKEKEARSKFRQIVSAVQYCHQKRIVHRDLKAENLLLDADMNIKIADFGFSNEFTVGGKLDTFCGSPPYAAPELFQGKKYDGPEVDVWSLGVILYTLVSGSLPFDGQNLKELRERVLRGKYRIPFYMSTDCENLLKRFLVLNPIKRGTLEQIMKDRWINAGHEEDELKPFVEPELDISDQKRIDIMVGMGYSQEEIQESLSKMKYDEITATYLLLGRKSSELDASDSSSSSNLSLAKVRPSSDLNNSTGQSPHHKVQRSVSSSQKQRRYSDHAGPAIPSVVAYPKRSQTSTADSDLKEDGISSRKSSGSAVGGKGIAPASPMLGNASNPNKADIPERKKSSTVPSSNTASGGMTRRNTYVCSERTTADRHSVIQNGKENSTIPDQRTPVASTHSISSAATPDRIRFPRGTASRSTFHGQPRERRTATYNGPPASPSLSHEATPLSQTRSRGSTNLFSKLTSKLTRRNMSFRFIKRLPTEYERNGRYEGSSRNVSAEQKDENKEAKPRSLRFTWSMKTTSSMDPGDMMREIRKVLDANNCDYEQRERFLLFCVHGDGHAENLVQWEMEVCKLPRLSLNGVRFKRISGTSIAFKNIASKIANELKL",
# GUC2D_HUMAN
"MTACARRAGGLPDPGLCGPAWWAPSLPRLPRALPRLPLLLLLLLLQPPALSAVFTVGVLGPWACDPIFSRARPDLAARLAAARLNRDPGLAGGPRFEVALLPEPCRTPGSLGAVSSALARVSGLVGPVNPAACRPAELLAEEAGIALVPWGCPWTQAEGTTAPAVTPAADALYALLRAFGWARVALVTAPQDLWVEAGRSLSTALRARGLPVASVTSMEPLDLSGAREALRKVRDGPRVTAVIMVMHSVLLGGEEQRYLLEAAEELGLTDGSLVFLPFDTIHYALSPGPEALAALANSSQLRRAHDAVLTLTRHCPSEGSVLDSLRRAQERRELPSDLNLQQVSPLFGTIYDAVFLLARGVAEARAAAGGRWVSGAAVARHIRDAQVPGFCGDLGGDEEPPFVLLDTDAAGDRLFATYMLDPARGSFLSAGTRMHFPRGGSAPGPDPSCWFDPNNICGGGLEPGLVFLGFLLVVGMGLAGAFLAHYVRHRLLHMQMVSGPNKIILTVDDITFLHPHGGTSRKVAQGSRSSLGARSMSDIRSGPSQHLDSPNIGVYEGDRVWLKKFPGDQHIAIRPATKTAFSKLQELRHENVALYLGLFLARGAEGPAALWEGNLAVVSEHCTRGSLQDLLAQREIKLDWMFKSSLLLDLIKGIRYLHHRGVAHGRLKSRNCIVDGRFVLKITDHGHGRLLEAQKVLPEPPRAEDQLWTAPELLRDPALERRGTLAGDVFSLAIIMQEVVCRSAPYAMLELTPEEVVQRVRSPPPLCRPLVSMDQAPVECILLMKQCWAEQPELRPSMDHTFDLFKNINKGRKTNIIDSMLRMLEQYSSNLEDLIRERTEELELEKQKTDRLLTQMLPPSVAEALKTGTPVEPEYFEQVTLYFSDIVGFTTISAMSEPIEVVDLLNDLYTLFDAIIGSHDVYKVETIGDAYMVASGLPQRNGQRHAAEIANMSLDILSAVGTFRMRHMPEVPVRIRIGLHSGPCVAGVVGLTMPRYCLFGDTVNTASRMESTGLPYRIHVNLSTVGILRALDSGYQVELRGRTELKGKGAEDTFWLVGRRGFNKPIPKPPDLQPGSSNHGISLQEIPPERRRKLEKARPGQFS",
# GUC2F_HUMAN
"MFLGLGRFSRLVLWFAAFRKLLGHHGLASAKFLWCLCLLSVMSLPQQVWTLPYKIGVVGPWACDSLFSKALPEVAARLAIERINRDPSFDLSYSFEYVILNEDCQTSRALSSFISHHQMASGFIGPTNPGYCEAASLLGNSWDKGIFSWACVNYELDNKISYPTFSRTLPSPIRVLVTVMKYFQWAHAGVISSDEDIWVHTANRVASALRSHGLPVGVVLTTGQDSQSMRKALQRIHQADRIRIIIMCMHSALIGGETQMHLLECAHDLKMTDGTYVFVPYDALLYSLPYKHTPYRVLRNNPKLREAYDAVLTITVESQEKTFYQAFTEAAARGEIPEKLEFDQVSPLFGTIYNSIYFIAQAMNNAMKENGQAGAASLVQHSRNMQFHGFNQLMRTDSNGNGISEYVILDTNLKEWELHSTYTVDMEMELLRFGGTPIHFPGGRPPRADAKCWFAEGKICHGGIDPAFAMMVCLTLLIALLSINGFAYFIRRRINKIQLIKGPNRILLTLEDVTFINPHFGSKRGSRASVSFQITSEVQSGRSPRLSFSSGSLTPATYENSNIAIYEGDWVWLKKFSLGDFGDLKSIKSRASDVFEMMKDLRHENINPLLGFFYDSGMFAIVTEFCSRGSLEDILTNQDVKLDWMFKSSLLLDLIKGMKYLHHREFVHGRLKSRNCVVDGRFVLKVTDYGFNDILEMLRLSEEESSMEELLWTAPELLRAPRGSRLGSFAGDVYSFAIIMQEVMVRGTPFCMMDLPAQEIINRLKKPPPVYRPVVPPEHAPPECLQLMKQCWAEAAEQRPTFDEIFNQFKTFNKGKKTNIIDSMLRMLEQYSSNLEDLIRERTEELEIEKQKTEKLLTQMLPPSVAESLKKGCTVEPEGFDLVTLYFSDIVGFTTISAMSEPIEVVDLLNDLYTLFDAIIGSHDVYKVETIGDAYMVASGLPKRNGSRHAAEIANMSLDILSSVGTFKMRHMPEVPVRIRIGLHSGPVVAGVVGLTMPRYCLFGDTVNTASRMESTGLPYRIHVSLSTVTILQNLSEGYEVELRGRTELKGKGTEETFWLIGKKGFMKPLPVPPPVDKDGQVGHGLQPVEIAAFQRRKAERQLVRNKP",
# DAPK1_HUMAN
"MTVFRQENVDDYYDTGEELGSGQFAVVKKCREKSTGLQYAAKFIKKRRTKSSRRGVSREDIEREVSILKEIQHPNVITLHEVYENKTDVILILELVAGGELFDFLAEKESLTEEEATEFLKQILNGVYYLHSLQIAHFDLKPENIMLLDRNVPKPRIKIIDFGLAHKIDFGNEFKNIFGTPEFVAPEIVNYEPLGLEADMWSIGVITYILLSGASPFLGDTKQETLANVSAVNYEFEDEYFSNTSALAKDFIRRLLVKDPKKRMTIQDSLQHPWIKPKDTQQALSRKASAVNMEKFKKFAARKKWKQSVRLISLCQRLSRSFLSRSNMSVARSDDTLDEEDSFVMKAIIHAINDDNVPGLQHLLGSLSNYDVNQPNKHGTPPLLIAAGCGNIQILQLLIKRGSRIDVQDKGGSNAVYWAARHGHVDTLKFLSENKCPLDVKDKSGEMALHVAARYGHADVAQLLCSFGSNPNIQDKEEETPLHCAAWHGYYSVAKALCEAGCNVNIKNREGETPLLTASARGYHDIVECLAEHGADLNACDKDGHIALHLAVRRCQMEVIKTLLSQGCFVDYQDRHGNTPLHVACKDGNMPIVVALCEANCNLDISNKYGRTPLHLAANNGILDVVRYLCLMGASVEALTTDGKTAEDLARSEQHEHVAGLLARLRKDTHRGLFIQQLRPTQNLQPRIKLKLFGHSGSGKTTLVESLKCGLLRSFFRRRRPRLSSTNSSRFPPSPLASKPTVSVSINNLYPGCENVSVRSRSMMFEPGLTKGMLEVFVAPTHHPHCSADDQSTKAIDIQNAYLNGVGDFSVWEFSGNPVYFCCYDYFAANDPTSIHVVVFSLEEPYEIQLNQVIFWLSFLKSLVPVEEPIAFGGKLKNPLQVVLVATHADIMNVPRPAGGEFGYDKDTSLLKEIRNRFGNDLHISNKLFVLDAGASGSKDMKVLRNHLQEIRSQIVSVCPPMTHLCEKIISTLPSWRKLNGPNQLMSLQQFVYDVQDQLNPLASEEDLRRIAQQLHSTGEINIMQSETVQDVLLLDPRWLCTNVLGKLLSVETPRALHHYRGRYTVEDIQRLVPDSDVEELLQILDAMDICARDLSSGTMVDVPALIKTDNLHRSWADEEDEVMVYGGVRIVPVEHLTPFPCGIFHKVQVNLCRWIHQQSTEGDADIRLWVNGCKLANRGAELLVLLVNHGQGIEVQVRGLETEKIKCCLLLDSVCSTIENVMATTLPGLLTVKHYLSPQQLREHHEPVMIYQPRDFFRAQTLKETSLTNTMGGYKESFSSIMCFGCHDVYSQASLGMDIHASDLNLLTRRKLSRLLDPPDPLGKDWCLLAMNLGLPDLVAKYNTSNGAPKDFLPSPLHALLREWTTYPESTVGTLMSKLRELGRRDAADFLLKASSVFKINLDGNGQEAYASSCNSGTSYNSISSVVSR",
# DAPK2_HUMAN
"MFQASMRSPNMEPFKQQKVEDFYDIGEELGSGQFAIVKKCREKSTGLEYAAKFIKKRQSRASRRGVSREEIEREVSILRQVLHHNVITLHDVYENRTDVVLILELVSGGELFDFLAQKESLSEEEATSFIKQILDGVNYLHTKKIAHFDLKPENIMLLDKNIPIPHIKLIDFGLAHEIEDGVEFKNIFGTPEFVAPEIVNYEPLGLEADMWSIGVITYILLSGASPFLGDTKQETLANITAVSYDFDEEFFSQTSELAKDFIRKLLVKETRKRLTIQEALRHPWITPVDNQQAMVRRESVVNLENFRKQYVRRRWKLSFSIVSLCNHLTRSLMKKVHLRPDEDLRNCESDTEEDIARRKALHPRRRSSTS",
# M3K12_HUMAN
"MACLHETRTPSPSFGGFVSTLSEASMRKLDPDTSDCTPEKDLTPTHVLQLHEQDAGGPGGAAGSPESRASRVRADEVRLQCQSGSGFLEGLFGCLRPVWTMIGKAYSTEHKQQQEDLWEVPFEEILDLQWVGSGAQGAVFLGRFHGEEVAVKKVRDLKETDIKHLRKLKHPNIITFKGVCTQAPCYCILMEFCAQGQLYEVLRAGRPVTPSLLVDWSMGIAGGMNYLHLHKIIHRDLKSPNMLITYDDVVKISDFGTSKELSDKSTKMSFAGTVAWMAPEVIRNEPVSEKVDIWSFGVVLWELLTGEIPYKDVDSSAIIWGVGSNSLHLPVPSSCPDGFKILLRQCWNSKPRNRPSFRQILLHLDIASADVLSTPQETYFKSQAEWREEVKLHFEKIKSEGTCLHRLEEELVMRRREELRHALDIREHYERKLERANNLYMELNALMLQLELKERELLRREQALERRCPGLLKPHPSRGLLHGNTMEKLIKKRNVPQKLSPHSKRPDILKTESLLPKLDAALSGVGLPGCPKGPPSPGRSRRGKTRHRKASAKGSCGDLPGLRTAVPPHEPGGPGSPGGLGGGPSAWEACPPALRGLHHDLLLRKMSSSSPDLLSAALGSRGRGATGGAGDPGSPPPARGDTPPSEGSAPGSTSPDSPGGAKGEPPPPVGPGEGVGLLGTGREGTSGRGGSRAGSQHLTPAALLYRAAVTRSQKRGISSEEEEGEVDSEVELTSSQRWPQSLNMRQSLSTFSSENPSDGEEGTASEPSPSGTPEVGSTNTDERPDERSDDMCSQGSEIPLDPPPSEVIPGPEPSSLPIPHQELLRERGPPNSEDSDCDSTELDNSNSVDALRPPASLPP",
# DMPK_HUMAN
"MSAEVRLRRLQQLVLDPGFLGLEPLLDLLLGVHQELGASELAQDKYVADFLQWAEPIVVRLKEVRLQRDDFEILKVIGRGAFSEVAVVKMKQTGQVYAMKIMNKWDMLKRGEVSCFREERDVLVNGDRRWITQLHFAFQDENYLYLVMEYYVGGDLLTLLSKFGERIPAEMARFYLAEIVMAIDSVHRLGYVHRDIKPDNILLDRCGHIRLADFGSCLKLRADGTVRSLVAVGTPDYLSPEILQAVGGGPGTGSYGPECDWWALGVFAYEMFYGQTPFYADSTAETYGKIVHYKEHLSLPLVDEGVPEEARDFIQRLLCPPETRLGRGGAGDFRTHPFFFGLDWDGLRDSVPPFTPDFEGATDTCNFDLVEDGLTAMVSGGGETLSDIREGAPLGVHLPFVGYSYSCMALRDSEVPGPTPMELEAEQLLEPHVQAPSLEPSVSPQDETAEVAVPAAVPAAEAEAEVTLRELQEALEEEVLTRQSLSREMEAIRTDNQNFASQLREAEARNRDLEAHVRQLQERMELLQAEGATAVTGVPSPRATDPPSHLDGPPAVAVGQCPLVGPGPMHRRHLLLPARVPRPGLSEALSLLLFAVVLSRAAALGCIGLVAHAGQLTAVWRRPGAARAP",
# MRCKG_HUMAN
"MERRLRALEQLARGEAGGCPGLDGLLDLLLALHHELSSGPLRRERSVAQFLSWASPFVSKVKELRLQRDDFEILKVIGRGAFGEVTVVRQRDTGQIFAMKMLHKWEMLKRAETACFREERDVLVKGDSRWVTTLHYAFQDEEYLYLVMDYYAGGDLLTLLSRFEDRLPPELAQFYLAEMVLAIHSLHQLGYVHRDVKPDNVLLDVNGHIRLADFGSCLRLNTNGMVDSSVAVGTPDYISPEILQAMEEGKGHYGPQCDWWSLGVCAYELLFGETPFYAESLVETYGKIMNHEDHLQFPPDVPDVPASAQDLIRQLLCRQEERLGRGGLDDFRNHPFFEGVDWERLASSTAPYIPELRGPMDTSNFDVDDDTLNHPGTLPPPSHGAFSGHHLPFVGFTYTSGSHSPESSSEAWAALERKLQCLEQEKVELSRKHQEALHAPTDHRELEQLRKEVQTLRDRLPEMLRDKASLSQTDGPPAGSPGQDSDLRQELDRLHRELAEGRAGLQAQEQELCRAQGQQEELLQRLQEAQEREAATASQTRALSSQLEEARAAQRELEAQVSSLSRQVTQLQGQWEQRLEESSQAKTIHTASETNGMGPPEGGPQEAQLRKEVAALREQLEQAHSHRPSGKEEALCQLQEENRRLSREQERLEAELAQEQESKQRLEGERRETESNWEAQLADILSWVNDEKVSRGYLQALATKMAEELESLRNVGTQTLPARPLDHQWKARRLQKMEASARLELQSALEAEIRAKQGLQERLTQVQEAQLQAERRLQEAEKQSQALQQELAMLREELRARGPVDTKPSNSLIPFLSFRSSEKDSAKDPGISGEATRHGGEPDLRPEGRRSLRMGAVFPRAPTANTASTEGLPAKPGSHTLRPRSFPSPTKCLRCTSLMLGLGRQGLGCDACGYFCHTTCAPQAPPCPVPPDLLRTALGVHPETGTGTAYEGFLSVPRPSGVRRGWQRVFAALSDSRLLLFDAPDLRLSPPSGALLQVLDLRDPQFSATPVLASDVIHAQSRDLPRIFRVTTSQLAVPPTTCTVLLLAESEGERERWLQVLGELQRLLLDARPRPRPVYTLKEAYDNGLPLLPHTLCAAILDQDRLALGTEEGLFVIHLRSNDIFQVGECRRVQQLTLSPSAGLLVVLCGRGPSVRLFALAELENIEVAGAKIPESRGCQVLAAGSILQARTPVLCVAVKRQVLCYQLGPGPGPWQRRIRELQAPATVQSLGLLGDRLCVGAAGGFALYPLLNEAAPLALGAGLVPEELPPSRGGLGEALGAVELSLSEFLLLFTTAGIYVDGAGRKSRGHELLWPAAPMGWGYAAPYLTVFSENSIDVFDVRRAEWVQTVPLKKVRPLNPEGSLFLYGTEKVRLTYLRNQLAEKDEFDIPDLTDNSRRQLFRTKSKRRFFFRVSEEQQKQQRREMLKDPFVRSKLISPPTNFNHLVHVGPANGRPGARDKSPAPEEKGRVARGSGPQRPHSFSEALRRPASMGSEGLGGDADPMKRKPWTSLSSESVSCPQGSLSPATSLMQVSERPRSLPLSPELESSP",
# PRKDC_HUMAN
"MAGSGAGVRCSLLRLQETLSAADRCGAALAGHQLIRGLGQECVLSSSPAVLALQTSLVFSRDFGLLVFVRKSLNSIEFRECREEILKFLCIFLEKMGQKIAPYSVEIKNTCTSVYTKDRAAKCKIPALDLLIKLLQTFRSSRLMDEFKIGELFSKFYGELALKKKIPDTVLEKVYELLGLLGEVHPSEMINNAENLFRAFLGELKTQMTSAVREPKLPVLAGCLKGLSSLLCNFTKSMEEDPQTSREIFNFVLKAIRPQIDLKRYAVPSAGLRLFALHASQFSTCLLDNYVSLFEVLLKWCAHTNVELKKAALSALESFLKQVSNMVAKNAEMHKNKLQYFMEQFYGIIRNVDSNNKELSIAIRGYGLFAGPCKVINAKDVDFMYVELIQRCKQMFLTQTDTGDDRVYQMPSFLQSVASVLLYLDTVPEVYTPVLEHLVVMQIDSFPQYSPKMQLVCCRAIVKVFLALAAKGPVLRNCISTVVHQGLIRICSKPVVLPKGPESESEDHRASGEVRTGKWKVPTYKDYVDLFRHLLSSDQMMDSILADEAFFSVNSSSESLNHLLYDEFVKSVLKIVEKLDLTLEIQTVGEQENGDEAPGVWMIPTSDPAANLHPAKPKDFSAFINLVEFCREILPEKQAEFFEPWVYSFSYELILQSTRLPLISGFYKLLSITVRNAKKIKYFEGVSPKSLKHSPEDPEKYSCFALFVKFGKEVAVKMKQYKDELLASCLTFLLSLPHNIIELDVRAYVPALQMAFKLGLSYTPLAEVGLNALEEWSIYIDRHVMQPYYKDILPCLDGYLKTSALSDETKNNWEVSALSRAAQKGFNKVVLKHLKKTKNLSSNEAISLEEIRIRVVQMLGSLGGQINKNLLTVTSSDEMMKSYVAWDREKRLSFAVPFREMKPVIFLDVFLPRVTELALTASDRQTKVAACELLHSMVMFMLGKATQMPEGGQGAPPMYQLYKRTFPVLLRLACDVDQVTRQLYEPLVMQLIHWFTNNKKFESQDTVALLEAILDGIVDPVDSTLRDFCGRCIREFLKWSIKQITPQQQEKSPVNTKSLFKRLYSLALHPNAFKRLGASLAFNNIYREFREEESLVEQFVFEALVIYMESLALAHADEKSLGTIQQCCDAIDHLCRIIEKKHVSLNKAKKRRLPRGFPPSASLCLLDLVKWLLAHCGRPQTECRHKSIELFYKFVPLLPGNRSPNLWLKDVLKEEGVSFLINTFEGGGCGQPSGILAQPTLLYLRGPFSLQATLCWLDLLLAALECYNTFIGERTVGALQVLGTEAQSSLLKAVAFFLESIAMHDIIAAEKCFGTGAAGNRTSPQEGERYNYSKCTVVVRIMEFTTTLLNTSPEGWKLLKKDLCNTHLMRVLVQTLCEPASIGFNIGDVQVMAHLPDVCVNLMKALKMSPYKDILETHLREKITAQSIEELCAVNLYGPDAQVDRSRLAAVVSACKQLHRAGLLHNILPSQSTDLHHSVGTELLSLVYKGIAPGDERQCLPSLDLSCKQLASGLLELAFAFGGLCERLVSLLLNPAVLSTASLGSSQGSVIHFSHGEYFYSLFSETINTELLKNLDLAVLELMQSSVDNTKMVSAVLNGMLDQSFRERANQKHQGLKLATTILQHWKKCDSWWAKDSPLETKMAVLALLAKILQIDSSVSFNTSHGSFPEVFTTYISLLADTKLDLHLKGQAVTLLPFFTSLTGGSLEELRRVLEQLIVAHFPMQSREFPPGTPRFNNYVDCMKKFLDALELSQSPMLLELMTEVLCREQQHVMEELFQSSFRRIARRGSCVTQVGLLESVYEMFRKDDPRLSFTRQSFVDRSLLTLLWHCSLDALREFFSTIVVDAIDVLKSRFTKLNESTFDTQITKKMGYYKILDVMYSRLPKDDVHAKESKINQVFHGSCITEGNELTKTLIKLCYDAFTENMAGENQLLERRRLYHCAAYNCAISVICCVFNELKFYQGFLFSEKPEKNLLIFENLIDLKRRYNFPVEVEVPMERKKKYIEIRKEAREAANGDSDGPSYMSSLSYLADSTLSEEMSQFDFSTGVQSYSYSSQDPRPATGRFRRREQRDPTVHDDVLELEMDELNRHECMAPLTALVKHMHRSLGPPQGEEDSVPRDLPSWMKFLHGKLGNPIVPLNIRLFLAKLVINTEEVFRPYAKHWLSPLLQLAASENNGGEGIHYMVVEIVATILSWTGLATPTGVPKDEVLANRLLNFLMKHVFHPKRAVFRHNLEIIKTLVECWKDCLSIPYRLIFEKFSGKDPNSKDNSVGIQLLGIVMANDLPPYDPQCGIQSSEYFQALVNNMSFVRYKEVYAAAAEVLGLILRYVMERKNILEESLCELVAKQLKQHQNTMEDKFIVCLNKVTKSFPPLADRFMNAVFFLLPKFHGVLKTLCLEVVLCRVEGMTELYFQLKSKDFVQVMRHRDDERQKVCLDIIYKMMPKLKPVELRELLNPVVEFVSHPSTTCREQMYNILMWIHDNYRDPESETDNDSQEIFKLAKDVLIQGLIDENPGLQLIIRNFWSHETRLPSNTLDRLLALNSLYSPKIEVHFLSLATNFLLEMTSMSPDYPNPMFEHPLSECEFQEYTIDSDWRFRSTVLTPMFVETQASQGTLQTRTQEGSLSARWPVAGQIRATQQQHDFTLTQTADGRSSFDWLTGSSTDPLVDHTSPSSDSLLFAHKRSERLQRAPLKSVGPDFGKKRLGLPGDEVDNKVKGAAGRTDLLRLRRRFMRDQEKLSLMYARKGVAEQKREKEIKSELKMKQDAQVVLYRSYRHGDLPDIQIKHSSLITPLQAVAQRDPIIAKQLFSSLFSGILKEMDKFKTLSEKNNITQKLLQDFNRFLNTTFSFFPPFVSCIQDISCQHAALLSLDPAAVSAGCLASLQQPVGIRLLEEALLRLLPAELPAKRVRGKARLPPDVLRWVELAKLYRSIGEYDVLRGIFTSEIGTKQITQSALLAEARSDYSEAAKQYDEALNKQDWVDGEPTEAEKDFWELASLDCYNHLAEWKSLEYCSTASIDSENPPDLNKIWSEPFYQETYLPYMIRSKLKLLLQGEADQSLLTFIDKAMHGELQKAILELHYSQELSLLYLLQDDVDRAKYYIQNGIQSFMQNYSSIDVLLHQSRLTKLQSVQALTEIQEFISFISKQGNLSSQVPLKRLLNTWTNRYPDAKMDPMNIWDDIITNRCFFLSKIEEKLTPLPEDNSMNVDQDGDPSDRMEVQEQEEDISSLIRSCKFSMKMKMIDSARKQNNFSLAMKLLKELHKESKTRDDWLVSWVQSYCRLSHCRSRSQGCSEQVLTVLKTVSLLDENNVSSYLSKNILAFRDQNILLGTTYRIIANALSSEPACLAEIEEDKARRILELSGSSSEDSEKVIAGLYQRAFQHLSEAVQAAEEEAQPPSWSCGPAAGVIDAYMTLADFCDQQLRKEEENASVIDSAELQAYPALVVEKMLKALKLNSNEARLKFPRLLQIIERYPEETLSLMTKEISSVPCWQFISWISHMVALLDKDQAVAVQHSVEEITDNYPQAIVYPFIISSESYSFKDTSTGHKNKEFVARIKSKLDQGGVIQDFINALDQLSNPELLFKDWSNDVRAELAKTPVNKKNIEKMYERMYAALGDPKAPGLGAFRRKFIQTFGKEFDKHFGKGGSKLLRMKLSDFNDITNMLLLKMNKDSKPPGNLKECSPWMSDFKVEFLRNELEIPGQYDGRGKPLPEYHVRIAGFDERVTVMASLRRPKRIIIRGHDEREHPFLVKGGEDLRQDQRVEQLFQVMNGILAQDSACSQRALQLRTYSVVPMTSRLGLIEWLENTVTLKDLLLNTMSQEEKAAYLSDPRAPPCEYKDWLTKMSGKHDVGAYMLMYKGANRTETVTSFRKRESKVPADLLKRAFVRMSTSPEAFLALRSHFASSHALICISHWILGIGDRHLNNFMVAMETGGVIGIDFGHAFGSATQFLPVPELMPFRLTRQFINLMLPMKETGLMYSIMVHALRAFRSDPGLLTNTMDVFVKEPSFDWKNFEQKMLKKGGSWIQEINVAEKNWYPRQKICYAKRKLAGANPAVITCDELLLGHEKAPAFRDYVAVARGSKDHNIRAQEPESGLSEETQVKCLMDQATDPNILGRTWEGWEPWM",
# DYR1B_HUMAN
"MAVPPGHGPFSGFPGPQEHTQVLPDVRLLPRRLPLAFRDATSAPLRKLSVDLIKTYKHINEVYYAKKKRRAQQAPPQDSSNKKEKKVLNHGYDDDNHDYIVRSGERWLERYEIDSLIGKGSFGQVVKAYDHQTQELVAIKIIKNKKAFLNQAQIELRLLELMNQHDTEMKYYIVHLKRHFMFRNHLCLVFELLSYNLYDLLRNTHFRGVSLNLTRKLAQQLCTALLFLATPELSIIHCDLKPENILLCNPKRSAIKIVDFGSSCQLGQRIYQYIQSRFYRSPEVLLGTPYDLAIDMWSLGCILVEMHTGEPLFSGSNEVDQMNRIVEVLGIPPAAMLDQAPKARKYFERLPGGGWTLRRTKELRKDYQGPGTRRLQEVLGVQTGGPGGRRAGEPGHSPADYLRFQDLVLRMLEYEPAARISPLGALQHGFFRRTADEATNTGPAGSSASTSPAPLDTCPSSSTASSISSSGGSSGSSSDNRTYRYSNRYCGGPGPPITDCEMNSPQVPPSQPLRPWAGGDVPHKTHQAPASASSLPGTGAQLPPQPRYLGRPPSPTSPPPPELMDVSLVGGPADCSPPHPAPAPQHPAASALRTRMTGGRPPLPPPDDPATLGPHLGLRGVPQSTAASS",
# DYRK2_HUMAN
"MLTRKPSAAAPAAYPTGRGGDSAVRQLQASPGLGAGATRSGVGTGPPSPIALPPLRASNAAAAAHTIGGSKHTMNDHLHVGSHAHGQIQVQQLFEDNSNKRTVLTTQPNGLTTVGKTGLPVVPERQLDSIHRRQGSSTSLKSMEGMGKVKATPMTPEQAMKQYMQKLTAFEHHEIFSYPEIYFLGLNAKKRQGMTGGPNNGGYDDDQGSYVQVPHDHVAYRYEVLKVIGKGSFGQVVKAYDHKVHQHVALKMVRNEKRFHRQAAEEIRILEHLRKQDKDNTMNVIHMLENFTFRNHICMTFELLSMNLYELIKKNKFQGFSLPLVRKFAHSILQCLDALHKNRIIHCDLKPENILLKQQGRSGIKVIDFGSSCYEHQRVYTYIQSRFYRAPEVILGARYGMPIDMWSLGCILAELLTGYPLLPGEDEGDQLACMIELLGMPSQKLLDASKRAKNFVSSKGYPRYCTVTTLSDGSVVLNGGRSRRGKLRGPPESREWGNALKGCDDPLFLDFLKQCLEWDPAVRMTPGQALRHPWLRRRLPKPPTGEKTSVKRITESTGAITSISKLPPPSSSASKLRTNLAQMTDANGNIQQRTVLPKLVS",
# DYRK4_HUMAN
"MPASELKASEIPFHPSIKTQDPKAEEKSPKKQKVTLTAAEALKLFKNQLSPYEQSEILGYAELWFLGLEAKKLDTAPEKFSKTSFDDEHGFYLKVLHDHIAYRYEVLETIGKGSFGQVAKCLDHKNNELVALKIIRNKKRFHQQALMELKILEALRKKDKDNTYNVVHMKDFFYFRNHFCITFELLGINLYELMKNNNFQGFSLSIVRRFTLSVLKCLQMLSVEKIIHCDLKPENIVLYQKGQASVKVIDFGSSCYEHQKVYTYIQSRFYRSPEVILGHPYDVAIDMWSLGCITAELYTGYPLFPGENEVEQLACIMEVLGLPPAGFIQTASRRQTFFDSKGFPKNITNNRGKKRYPDSKDLTMVLKTYDTSFLDFLRRCLVWEPSLRMTPDQALKHAWIHQSRNLKPQPRPQTLRKSNSFFPSETRKDKVQGCHHSSRKADEITKETTEKTKDSPTKHVQHSGDQQDCLQHGADTVQLPQLVDAPKKSEAAVGAEVSMTSPGQSKNFSLKNTNVLPPIV",
# EF2K_HUMAN
"MADEDLIFRLEGVDGGQSPRAGHDGDSDGDSDDEEGYFICPITDDPSSNQNVNSKVNKYYSNLTKSERYSSSGSPANSFHFKEAWKHAIQKAKHMPDPWAEFHLEDIATERATRHRYNAVTGEWLDDEVLIKMASQPFGRGAMRECFRTKKLSNFLHAQQWKGASNYVAKRYIEPVDRDVYFEDVRLQMEAKLWGEEYNRHKPPKQVDIMQMCIIELKDRPGKPLFHLEHYIEGKYIKYNSNSGFVRDDNIRLTPQAFSHFTFERSGHQLIVVDIQGVGDLYTDPQIHTETGTDFGDGNLGVRGMALFFYSHACNRICESMGLAPFDLSPRERDAVNQNTKLLQSAKTILRGTEEKCGSPQVRTLSGSRPPLLRPLSENSGDENMSDVTFDSLPSSPSSATPHSQKLDHLHWPVFSDLDNMASRDHDHLDNHRESENSGDSGYPSEKRGELDDPEPREHGHSYSNRKYESDEDSLGSSGRVCVEKWNLLNSSRLHLPRASAVALEVQRLNALDLEKKIGKSILGKVHLAMVRYHEGGRFCEKGEEWDQESAVFHLEHAANLGELEAIVGLGLMYSQLPHHILADVSLKETEENKTKGFDYLLKAAEAGDRQSMILVARAFDSGQNLSPDRCQDWLEALHWYNTALEMTDCDEGGEYDGMQDEPRYMMLAREAEMLFTGGYGLEKDPQRSGDLYTQAAEAAMEAMKGRLANQYYQKAEEAWAQMEE",
# EGFR_HUMAN
"MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRALMDEEDMDDVVDADEYLIPQQGFFSSPSTSRTPLLSSLSATSNNSTVACIDRNGLQSCPIKEDSFLQRYSSDPTGALTEDSIDDTFLPVPEYINQSVPKRPAGSVQNPVYHNQPLNPAPSRDPHYQDPHSTAVGNPEYLNTVQPTCVNSTFDSPAHWAQKGSHQISLDNPDYQQDFFPKEAKPNGIFKGSTAENAEYLRVAPQSSEFIGA",
# E2AK2_HUMAN
"MAGDLSAGFFMEELNTYRQKQGVVLKYQELPNSGPPHDRRFTFQVIIDGREFPEGEGRSKKEAKNAAAKLAVEILNKEKKAVSPLLLTTTNSSEGLSMGNYIGLINRIAQKKRLTVNYEQCASGVHGPEGFHYKCKMGQKEYSIGTGSTKQEAKQLAAKLAYLQILSEETSVKSDYLSSGSFATTCESQSNSLVTSTLASESSSEGDFSADTSEINSNSDSLNSSSLLMNGLRNNQRKAKRSLAPRFDLPDMKETKYTVDKRFGMDFKEIELIGSGGFGQVFKAKHRIDGKTYVIKRVKYNNEKAEREVKALAKLDHVNIVHYNGCWDGFDYDPETSDDSLESSDYDPENSKNSSRSKTKCLFIQMEFCDKGTLEQWIEKRRGEKLDKVLALELFEQITKGVDYIHSKKLIHRDLKPSNIFLVDTKQVKIGDFGLVTSLKNDGKRTRSKGTLRYMSPEQISSQDYGKEVDLYALGLILAELLHVCDTAFETSKFFTDLRDGIISDIFDKKEKTLLQKLLSKKPEDRPNTSEILRTLTVWKKSPEKNERHTC",
# MARK2_HUMAN
"MSSARTPLPTLNERDTEQPTLGHLDSKPSSKSNMIRGRNSATSADEQPHIGNYRLLKTIGKGNFAKVKLARHILTGKEVAVKIIDKTQLNSSSLQKLFREVRIMKVLNHPNIVKLFEVIETEKTLYLVMEYASGGEVFDYLVAHGRMKEKEARAKFRQIVSAVQYCHQKFIVHRDLKAENLLLDADMNIKIADFGFSNEFTFGNKLDTFCGSPPYAAPELFQGKKYDGPEVDVWSLGVILYTLVSGSLPFDGQNLKELRERVLRGKYRIPFYMSTDCENLLKKFLILNPSKRGTLEQIMKDRWMNVGHEDDELKPYVEPLPDYKDPRRTELMVSMGYTREEIQDSLVGQRYNEVMATYLLLGYKSSELEGDTITLKPRPSADLTNSSAPSPSHKVQRSVSANPKQRRFSDQAAGPAIPTSNSYSKKTQSNNAENKRPEEDRESGRKASSTAKVPASPLPGLERKKTTPTPSTNSVLSTSTNRSRNSPLLERASLGQASIQNGKDSLTMPGSRASTASASAAVSAARPRQHQKSMSASVHPNKASGLPPTESNCEVPRPSTAPQRVPVASPSAHNISSSGGAPDRTNFPRGVSSRSTFHAGQLRQVRDQQNLPYGVTPASPSGHSQGRRGASGSIFSKFTSKFVRRNLSFRFARRNLNEPESKDRVETLRPHVVGSGGNDKEKEEFREAKPRSLRFTWSMKTTSSMEPNEMMREIRKVLDANSCQSELHEKYMLLCMHGTPGHEDFVQWEMEVCKLPRLSLNGVRFKRISGTSMAFKNIASKIANELKL",
# EPHA1_HUMAN
"MERRWPLGLGLVLLLCAPLPPGARAKEVTLMDTSKAQGELGWLLDPPKDGWSEQQQILNGTPLYMYQDCPMQGRRDTDHWLRSNWIYRGEEASRVHVELQFTVRDCKSFPGGAGPLGCKETFNLLYMESDQDVGIQLRRPLFQKVTTVAADQSFTIRDLVSGSVKLNVERCSLGRLTRRGLYLAFHNPGACVALVSVRVFYQRCPETLNGLAQFPDTLPGPAGLVEVAGTCLPHARASPRPSGAPRMHCSPDGEWLVPVGRCHCEPGYEEGGSGEACVACPSGSYRMDMDTPHCLTCPQQSTAESEGATICTCESGHYRAPGEGPQVACTGPPSAPRNLSFSASGTQLSLRWEPPADTGGRQDVRYSVRCSQCQGTAQDGGPCQPCGVGVHFSPGARGLTTPAVHVNGLEPYANYTFNVEAQNGVSGLGSSGHASTSVSISMGHAESLSGLSLRLVKKEPRQLELTWAGSRPRSPGANLTYELHVLNQDEERYQMVLEPRVLLTELQPDTTYIVRVRMLTPLGPGPFSPDHEFRTSPPVSRGLTGGEIVAVIFGLLLGAALLLGILVFRSRRAQRQRQQRQRDRATDVDREDKLWLKPYVDLQAYEDPAQGALDFTRELDPAWLMVDTVIGEGEFGEVYRGTLRLPSQDCKTVAIKTLKDTSPGGQWWNFLREATIMGQFSHPHILHLEGVVTKRKPIMIITEFMENGALDAFLREREDQLVPGQLVAMLQGIASGMNYLSNHNYVHRDLAARNILVNQNLCCKVSDFGLTRLLDDFDGTYETQGGKIPIRWTAPEAIAHRIFTTASDVWSFGIVMWEVLSFGDKPYGEMSNQEVMKSIEDGYRLPPPVDCPAPLYELMKNCWAYDRARRPHFQKLQAHLEQLLANPHSLRTIANFDPRMTLRLPSLSGSDGIPYRTVSEWLESIRMKRYILHFHSAGLDTMECVLELTAEDLTQMGITLPGHQKRILCSIQGFKD",
# EPHA2_HUMAN
"MELQAARACFALLWGCALAAAAAAQGKEVVLLDFAAAGGELGWLTHPYGKGWDLMQNIMNDMPIYMYSVCNVMSGDQDNWLRTNWVYRGEAERIFIELKFTVRDCNSFPGGASSCKETFNLYYAESDLDYGTNFQKRLFTKIDTIAPDEITVSSDFEARHVKLNVEERSVGPLTRKGFYLAFQDIGACVALLSVRVYYKKCPELLQGLAHFPETIAGSDAPSLATVAGTCVDHAVVPPGGEEPRMHCAVDGEWLVPIGQCLCQAGYEKVEDACQACSPGFFKFEASESPCLECPEHTLPSPEGATSCECEEGFFRAPQDPASMPCTRPPSAPHYLTAVGMGAKVELRWTPPQDSGGREDIVYSVTCEQCWPESGECGPCEASVRYSEPPHGLTRTSVTVSDLEPHMNYTFTVEARNGVSGLVTSRSFRTASVSINQTEPPKVRLEGRSTTSLSVSWSIPPPQQSRVWKYEVTYRKKGDSNSYNVRRTEGFSVTLDDLAPDTTYLVQVQALTQEGQGAGSKVHEFQTLSPEGSGNLAVIGGVAVGVVLLLVLAGVGFFIHRRRKNQRARQSPEDVYFSKSEQLKPLKTYVDPHTYEDPNQAVLKFTTEIHPSCVTRQKVIGAGEFGEVYKGMLKTSSGKKEVPVAIKTLKAGYTEKQRVDFLGEAGIMGQFSHHNIIRLEGVISKYKPMMIITEYMENGALDKFLREKDGEFSVLQLVGMLRGIAAGMKYLANMNYVHRDLAARNILVNSNLVCKVSDFGLSRVLEDDPEATYTTSGGKIPIRWTAPEAISYRKFTSASDVWSFGIVMWEVMTYGERPYWELSNHEVMKAINDGFRLPTPMDCPSAIYQLMMQCWQQERARRPKFADIVSILDKLIRAPDSLKTLADFDPRVSIRLPSTSGSEGVPFRTVSEWLESIKMQQYTEHFMAAGYTAIEKVVQMTNDDIKRIGVRLPGHQKRIAYSLLGLKDQVNTVGIPI",
# EPHA3_HUMAN
"MDCQLSILLLLSCSVLDSFGELIPQPSNEVNLLDSKTIQGELGWISYPSHGWEEISGVDEHYTPIRTYQVCNVMDHSQNNWLRTNWVPRNSAQKIYVELKFTLRDCNSIPLVLGTCKETFNLYYMESDDDHGVKFREHQFTKIDTIAADESFTQMDLGDRILKLNTEIREVGPVNKKGFYLAFQDVGACVALVSVRVYFKKCPFTVKNLAMFPDTVPMDSQSLVEVRGSCVNNSKEEDPPRMYCSTEGEWLVPIGKCSCNAGYEERGFMCQACRPGFYKALDGNMKCAKCPPHSSTQEDGSMNCRCENNYFRADKDPPSMACTRPPSSPRNVISNINETSVILDWSWPLDTGGRKDVTFNIICKKCGWNIKQCEPCSPNVRFLPRQFGLTNTTVTVTDLLAHTNYTFEIDAVNGVSELSSPPRQFAAVSITTNQAAPSPVLTIKKDRTSRNSISLSWQEPEHPNGIILDYEVKYYEKQEQETSYTILRARGTNVTISSLKPDTIYVFQIRARTAAGYGTNSRKFEFETSPDSFSISGESSQVVMIAISAAVAIILLTVVIYVLIGRFCGYKSKHGADEKRLHFGNGHLKLPGLRTYVDPHTYEDPTQAVHEFAKELDATNISIDKVVGAGEFGEVCSGRLKLPSKKEISVAIKTLKVGYTEKQRRDFLGEASIMGQFDHPNIIRLEGVVTKSKPVMIVTEYMENGSLDSFLRKHDAQFTVIQLVGMLRGIASGMKYLSDMGYVHRDLAARNILINSNLVCKVSDFGLSRVLEDDPEAAYTTRGGKIPIRWTSPEAIAYRKFTSASDVWSYGIVLWEVMSYGERPYWEMSNQDVIKAVDEGYRLPPPMDCPAALYQLMLDCWQKDRNNRPKFEQIVSILDKLIRNPGSLKIITSAAARPSNLLLDQSNVDITTFRTTGDWLNGVWTAHCKEIFTGVEYSSCDTIAKISTDDMKKVGVTVVGPQKKIISSIKALETQSKNGPVPV",
# EPHA4_HUMAN
"MAGIFYFALFSCLFGICDAVTGSRVYPANEVTLLDSRSVQGELGWIASPLEGGWEEVSIMDEKNTPIRTYQVCNVMEPSQNNWLRTDWITREGAQRVYIEIKFTLRDCNSLPGVMGTCKETFNLYYYESDNDKERFIRENQFVKIDTIAADESFTQVDIGDRIMKLNTEIRDVGPLSKKGFYLAFQDVGACIALVSVRVFYKKCPLTVRNLAQFPDTITGADTSSLVEVRGSCVNNSEEKDVPKMYCGADGEWLVPIGNCLCNAGHEERSGECQACKIGYYKALSTDATCAKCPPHSYSVWEGATSCTCDRGFFRADNDAASMPCTRPPSAPLNLISNVNETSVNLEWSSPQNTGGRQDISYNVVCKKCGAGDPSKCRPCGSGVHYTPQQNGLKTTKVSITDLLAHTNYTFEIWAVNGVSKYNPNPDQSVSVTVTTNQAAPSSIALVQAKEVTRYSVALAWLEPDRPNGVILEYEVKYYEKDQNERSYRIVRTAARNTDIKGLNPLTSYVFHVRARTAAGYGDFSEPLEVTTNTVPSRIIGDGANSTVLLVSVSGSVVLVVILIAAFVISRRRSKYSKAKQEADEEKHLNQGVRTYVDPFTYEDPNQAVREFAKEIDASCIKIEKVIGVGEFGEVCSGRLKVPGKREICVAIKTLKAGYTDKQRRDFLSEASIMGQFDHPNIIHLEGVVTKCKPVMIITEYMENGSLDAFLRKNDGRFTVIQLVGMLRGIGSGMKYLSDMSYVHRDLAARNILVNSNLVCKVSDFGMSRVLEDDPEAAYTTRGGKIPIRWTAPEAIAYRKFTSASDVWSYGIVMWEVMSYGERPYWDMSNQDVIKAIEEGYRLPPPMDCPIALHQLMLDCWQKERSDRPKFGQIVNMLDKLIRNPNSLKRTGTESSRPNTALLDPSSPEFSAVVSVGDWLQAIKMDRYKDNFTAAGYTTLEAVVHVNQEDLARIGITAITHQNKILSSVQAMRTQMQQMHGRMVPV",
# EPHA5_HUMAN
"MRGSGPRGAGRRRPPSGGGDTPITPASLAGCYSAPRRAPLWTCLLLCAALRTLLASPSNEVNLLDSRTVMGDLGWIAFPKNGWEEIGEVDENYAPIHTYQVCKVMEQNQNNWLLTSWISNEGASRIFIELKFTLRDCNSLPGGLGTCKETFNMYYFESDDQNGRNIKENQYIKIDTIAADESFTELDLGDRVMKLNTEVRDVGPLSKKGFYLAFQDVGACIALVSVRVYYKKCPSVVRHLAVFPDTITGADSSQLLEVSGSCVNHSVTDEPPKMHCSAEGEWLVPIGKCMCKAGYEEKNGTCQVCRPGFFKASPHIQSCGKCPPHSYTHEEASTSCVCEKDYFRRESDPPTMACTRPPSAPRNAISNVNETSVFLEWIPPADTGGRKDVSYYIACKKCNSHAGVCEECGGHVRYLPRQSGLKNTSVMMVDLLAHTNYTFEIEAVNGVSDLSPGARQYVSVNVTTNQAAPSPVTNVKKGKIAKNSISLSWQEPDRPNGIILEYEIKYFEKDQETSYTIIKSKETTITAEGLKPASVYVFQIRARTAAGYGVFSRRFEFETTPVFAASSDQSQIPVIAVSVTVGVILLAVVIGVLLSGSCCECGCGRASSLCAVAHPSLIWRCGYSKAKQDPEEEKMHFHNGHIKLPGVRTYIDPHTYEDPNQAVHEFAKEIEASCITIERVIGAGEFGEVCSGRLKLPGKRELPVAIKTLKVGYTEKQRRDFLGEASIMGQFDHPNIIHLEGVVTKSKPVMIVTEYMENGSLDTFLKKNDGQFTVIQLVGMLRGISAGMKYLSDMGYVHRDLAARNILINSNLVCKVSDFGLSRVLEDDPEAAYTTRGGKIPIRWTAPEAIAFRKFTSASDVWSYGIVMWEVVSYGERPYWEMTNQDVIKAVEEGYRLPSPMDCPAALYQLMLDCWQKERNSRPKFDEIVNMLDKLIRNPSSLKTLVNASCRVSNLLAEHSPLGSGAYRSVGEWLEAIKMGRYTEIFMENGYSSMDAVAQVTLEDLRRLGVTLVGHQKKIMNSLQEMKVQLVNGMVPL",
# EPHA8_HUMAN
"MAPARGRLPPALWVVTAAAAAATCVSAARGEVNLLDTSTIHGDWGWLTYPAHGWDSINEVDESFQPIHTYQVCNVMSPNQNNWLRTSWVPRDGARRVYAEIKFTLRDCNSMPGVLGTCKETFNLYYLESDRDLGASTQESQFLKIDTIAADESFTGADLGVRRLKLNTEVRSVGPLSKRGFYLAFQDIGACLAILSLRIYYKKCPAMVRNLAAFSEAVTGADSSSLVEVRGQCVRHSEERDTPKMYCSAEGEWLVPIGKCVCSAGYEERRDACVACELGFYKSAPGDQLCARCPPHSHSAAPAAQACHCDLSYYRAALDPPSSACTRPPSAPVNLISSVNGTSVTLEWAPPLDPGGRSDITYNAVCRRCPWALSRCEACGSGTRFVPQQTSLVQASLLVANLLAHMNYSFWIEAVNGVSDLSPEPRRAAVVNITTNQAAPSQVVVIRQERAGQTSVSLLWQEPEQPNGIILEYEIKYYEKDKEMQSYSTLKAVTTRATVSGLKPGTRYVFQVRARTSAGCGRFSQAMEVETGKPRPRYDTRTIVWICLTLITGLVVLLLLLICKKRHCGYSKAFQDSDEEKMHYQNGQAPPPVFLPLHHPPGKLPEPQFYAEPHTYEEPGRAGRSFTREIEASRIHIEKIIGSGDSGEVCYGRLRVPGQRDVPVAIKALKAGYTERQRRDFLSEASIMGQFDHPNIIRLEGVVTRGRLAMIVTEYMENGSLDTFLRTHDGQFTIMQLVGMLRGVGAGMRYLSDLGYVHRDLAARNVLVDSNLVCKVSDFGLSRVLEDDPDAAYTTTGGKIPIRWTAPEAIAFRTFSSASDVWSFGVVMWEVLAYGERPYWNMTNRDVISSVEEGYRLPAPMGCPHALHQLMLDCWHKDRAQRPRFSQIVSVLDALIRSPESLRATATVSRCPPPAFVRSCFDLRGGSGGGGGLTVGDWLDSIRMGRYRDHFAAGGYSSLGMVLRMNAQDVRALGITLMGHQKKILGSIQTMRAQLTSTQGPRRHL",
# EPHB1_HUMAN
"MALDYLLLLLLASAVAAMEETLMDTRTATAELGWTANPASGWEEVSGYDENLNTIRTYQVCNVFEPNQNNWLLTTFINRRGAHRIYTEMRFTVRDCSSLPNVPGSCKETFNLYYYETDSVIATKKSAFWSEAPYLKVDTIAADESFSQVDFGGRLMKVNTEVRSFGPLTRNGFYLAFQDYGACMSLLSVRVFFKKCPSIVQNFAVFPETMTGAESTSLVIARGTCIPNAEEVDVPIKLYCNGDGEWMVPIGRCTCKPGYEPENSVACKACPAGTFKASQEAEGCSHCPSNSRSPAEASPICTCRTGYYRADFDPPEVACTSVPSGPRNVISIVNETSIILEWHPPRETGGRDDVTYNIICKKCRADRRSCSRCDDNVEFVPRQLGLTECRVSISSLWAHTPYTFDIQAINGVSSKSPFPPQHVSVNITTNQAAPSTVPIMHQVSATMRSITLSWPQPEQPNGIILDYEIRYYEKEHNEFNSSMARSQTNTARIDGLRPGMVYVVQVRARTVAGYGKFSGKMCFQTLTDDDYKSELREQLPLIAGSAAAGVVFVVSLVAISIVCSRKRAYSKEAVYSDKLQHYSTGRGSPGMKIYIDPFTYEDPNEAVREFAKEIDVSFVKIEEVIGAGEFGEVYKGRLKLPGKREIYVAIKTLKAGYSEKQRRDFLSEASIMGQFDHPNIIRLEGVVTKSRPVMIITEFMENGALDSFLRQNDGQFTVIQLVGMLRGIAAGMKYLAEMNYVHRDLAARNILVNSNLVCKVSDFGLSRYLQDDTSDPTYTSSLGGKIPVRWTAPEAIAYRKFTSASDVWSYGIVMWEVMSFGERPYWDMSNQDVINAIEQDYRLPPPMDCPAALHQLMLDCWQKDRNSRPRFAEIVNTLDKMIRNPASLKTVATITAVPSQPLLDRSIPDFTAFTTVDDWLSAIKMVQYRDSFLTAGFTSLQLVTQMTSEDLLRIGITLAGHQKKILNSIHSMRVQISQSPTAMA",
# EPHB2_HUMAN
"MALRRLGAALLLLPLLAAVEETLMDSTTATAELGWMVHPPSGWEEVSGYDENMNTIRTYQVCNVFESSQNNWLRTKFIRRRGAHRIHVEMKFSVRDCSSIPSVPGSCKETFNLYYYEADFDSATKTFPNWMENPWVKVDTIAADESFSQVDLGGRVMKINTEVRSFGPVSRSGFYLAFQDYGGCMSLIAVRVFYRKCPRIIQNGAIFQETLSGAESTSLVAARGSCIANAEEVDVPIKLYCNGDGEWLVPIGRCMCKAGFEAVENGTVCRGCPSGTFKANQGDEACTHCPINSRTTSEGATNCVCRNGYYRADLDPLDMPCTTIPSAPQAVISSVNETSLMLEWTPPRDSGGREDLVYNIICKSCGSGRGACTRCGDNVQYAPRQLGLTEPRIYISDLLAHTQYTFEIQAVNGVTDQSPFSPQFASVNITTNQAAPSAVSIMHQVSRTVDSITLSWSQPDQPNGVILDYELQYYEKELSEYNATAIKSPTNTVTVQGLKAGAIYVFQVRARTVAGYGRYSGKMYFQTMTEAEYQTSIQEKLPLIIGSSAAGLVFLIAVVVIAIVCNRRGFERADSEYTDKLQHYTSGHMTPGMKIYIDPFTYEDPNEAVREFAKEIDISCVKIEQVIGAGEFGEVCSGHLKLPGKREIFVAIKTLKSGYTEKQRRDFLSEASIMGQFDHPNVIHLEGVVTKSTPVMIITEFMENGSLDSFLRQNDGQFTVIQLVGMLRGIAAGMKYLADMNYVHRDLAARNILVNSNLVCKVSDFGLSRFLEDDTSDPTYTSALGGKIPIRWTAPEAIQYRKFTSASDVWSYGIVMWEVMSYGERPYWDMTNQDVINAIEQDYRLPPPMDCPSALHQLMLDCWQKDRNHRPKFGQIVNTLDKMIRNPNSLKAMAPLSSGINLPLLDRTIPDYTSFNTVDEWLEAIKMGQYKESFANAGFTSFDVVSQMMMEDILRVGVTLAGHQKKILNSIQVMRAQMNQIQSVEGQPLARRPRATGRTKRCQPRDVTKKTCNSNDGKKKGMGKKKTDPGRGREIQGIFFKEDSHKESNDCSCGG",
# EPHB3_HUMAN
"MARARPPPPPSPPPGLLPLLPPLLLLPLLLLPAGCRALEETLMDTKWVTSELAWTSHPESGWEEVSGYDEAMNPIRTYQVCNVRESSQNNWLRTGFIWRRDVQRVYVELKFTVRDCNSIPNIPGSCKETFNLFYYEADSDVASASSPFWMENPYVKVDTIAPDESFSRLDAGRVNTKVRSFGPLSKAGFYLAFQDQGACMSLISVRAFYKKCASTTAGFALFPETLTGAEPTSLVIAPGTCIPNAVEVSVPLKLYCNGDGEWMVPVGACTCATGHEPAAKESQCRPCPPGSYKAKQGEGPCLPCPPNSRTTSPAASICTCHNNFYRADSDSADSACTTVPSPPRGVISNVNETSLILEWSEPRDLGGRDDLLYNVICKKCHGAGGASACSRCDDNVEFVPRQLGLTERRVHISHLLAHTRYTFEVQAVNGVSGKSPLPPRYAAVNITTNQAAPSEVPTLRLHSSSGSSLTLSWAPPERPNGVILDYEMKYFEKSEGIASTVTSQMNSVQLDGLRPDARYVVQVRARTVAGYGQYSRPAEFETTSERGSGAQQLQEQLPLIVGSATAGLVFVVAVVVIAIVCLRKQRHGSDSEYTEKLQQYIAPGMKVYIDPFTYEDPNEAVREFAKEIDVSCVKIEEVIGAGEFGEVCRGRLKQPGRREVFVAIKTLKVGYTERQRRDFLSEASIMGQFDHPNIIRLEGVVTKSRPVMILTEFMENCALDSFLRLNDGQFTVIQLVGMLRGIAAGMKYLSEMNYVHRDLAARNILVNSNLVCKVSDFGLSRFLEDDPSDPTYTSSLGGKIPIRWTAPEAIAYRKFTSASDVWSYGIVMWEVMSYGERPYWDMSNQDVINAVEQDYRLPPPMDCPTALHQLMLDCWVRDRNLRPKFSQIVNTLDKLIRNAASLKVIASAQSGMSQPLLDRTVPDYTTFTTVGDWLDAIKMGRYKESFVSAGFASFDLVAQMTAEDLLRIGVTLAGHQKKILSSIQDMRLQMNQTLPVQV",
# EPHB4_HUMAN
"MELRVLLCWASLAAALEETLLNTKLETADLKWVTFPQVDGQWEELSGLDEEQHSVRTYEVCDVQRAPGQAHWLRTGWVPRRGAVHVYATLRFTMLECLSLPRAGRSCKETFTVFYYESDADTATALTPAWMENPYIKVDTVAAEHLTRKRPGAEATGKVNVKTLRLGPLSKAGFYLAFQDQGACMALLSLHLFYKKCAQLTVNLTRFPETVPRELVVPVAGSCVVDAVPAPGPSPSLYCREDGQWAEQPVTGCSCAPGFEAAEGNTKCRACAQGTFKPLSGEGSCQPCPANSHSNTIGSAVCQCRVGYFRARTDPRGAPCTTPPSAPRSVVSRLNGSSLHLEWSAPLESGGREDLTYALRCRECRPGGSCAPCGGDLTFDPGPRDLVEPWVVVRGLRPDFTYTFEVTALNGVSSLATGPVPFEPVNVTTDREVPPAVSDIRVTRSSPSSLSLAWAVPRAPSGAVLDYEVKYHEKGAEGPSSVRFLKTSENRAELRGLKRGASYLVQVRARSEAGYGPFGQEHHSQTQLDESEGWREQLALIAGTAVVGVVLVLVVIVVAVLCLRKQSNGREAEYSDKHGQYLIGHGTKVYIDPFTYEDPNEAVREFAKEIDVSYVKIEEVIGAGEFGEVCRGRLKAPGKKESCVAIKTLKGGYTERQRREFLSEASIMGQFEHPNIIRLEGVVTNSMPVMILTEFMENGALDSFLRLNDGQFTVIQLVGMLRGIASGMRYLAEMSYVHRDLAARNILVNSNLVCKVSDFGLSRFLEENSSDPTYTSSLGGKIPIRWTAPEAIAFRKFTSASDAWSYGIVMWEVMSFGERPYWDMSNQDVINAIEQDYRLPPPPDCPTSLHQLMLDCWQKDRNARPRFPQVVSALDKMIRNPASLKIVARENGGASHPLLDQRQPHYSAFGSVGEWLRAIKMGRYEESFAAAGFGSFELVSQISAEDLLRIGVTLAGHQKKILASVQHMKSQAKPGTPGGTGGPAPQY",
# EPHB6_HUMAN
"MATEGAAQLGNRVAGMVCSLWVLLLVSSVLALEEVLLDTTGETSEIGWLTYPPGGWDEVSVLDDQRRLTRTFEACHVAGAPPGTGQDNWLQTHFVERRGAQRAHIRLHFSVRACSSLGVSGGTCRETFTLYYRQAEEPDSPDSVSSWHLKRWTKVDTIAADESFPSSSSSSSSSSSAAWAVGPHGAGQRAGLQLNVKERSFGPLTQRGFYVAFQDTGACLALVAVRLFSYTCPAVLRSFASFPETQASGAGGASLVAAVGTCVAHAEPEEDGVGGQAGGSPPRLHCNGEGKWMVAVGGCRCQPGYQPARGDKACQACPRGLYKSSAGNAPCSPCPARSHAPNPAAPVCPCLEGFYRASSDPPEAPCTGPPSAPQELWFEVQGSALMLHWRLPRELGGRGDLLFNVVCKECEGRQEPASGGGGTCHRCRDEVHFDPRQRGLTESRVLVGGLRAHVPYILEVQAVNGVSELSPDPPQAAAINVSTSHEVPSAVPVVHQVSRASNSITVSWPQPDQTNGNILDYQLRYYDQAEDESHSFTLTSETNTATVTQLSPGHIYGFQVRARTAAGHGPYGGKVYFQTLPQGELSSQLPERLSLVIGSILGALAFLLLAAITVLAVVFQRKRRGTGYTEQLQQYSSPGLGVKYYIDPSTYEDPCQAIRELAREVDPAYIKIEEVIGTGSFGEVRQGRLQPRGRREQTVAIQALWAGGAESLQMTFLGRAAVLGQFQHPNILRLEGVVTKSRPLMVLTEFMELGPLDSFLRQREGQFSSLQLVAMQRGVAAAMQYLSSFAFVHRSLSAHSVLVNSHLVCKVARLGHSPQGPSCLLRWAAPEVIAHGKHTTSSDVWSFGILMWEVMSYGERPYWDMSEQEVLNAIEQEFRLPPPPGCPPGLHLLMLDTWQKDRARRPHFDQLVAAFDKMIRKPDTLQAGGDPGERPSQALLTPVALDFPCLDSPQAWLSAIGLECYQDNFSKFGLCTFSDVAQLSLEDLPALGITLAGHQKKLLHHIQLLQQHLRQQGSVEV",
# MK03_HUMAN
"MAAAAAQGGGGGEPRRTEGVGPGVPGEVEMVKGQPFDVGPRYTQLQYIGEGAYGMVSSAYDHVRKTRVAIKKISPFEHQTYCQRTLREIQILLRFRHENVIGIRDILRASTLEAMRDVYIVQDLMETDLYKLLKSQQLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLINTTCDLKICDFGLARIADPEHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINMKARNYLQSLPSKTKVAWAKLFPKSDSKALDLLDRMLTFNPNKRITVEEALAHPYLEQYYDPTDEPVAEEPFTFAMELDDLPKERLKELIFQETARFQPGVLEAP",
# MK01_HUMAN
"MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNVNKVRVAIKKISPFEHQTYCQRTLREIKILLRFRHENIIGINDIIRAPTIEQMKDVYIVQDLMETDLYKLLKTQHLSNDHICYFLYQILRGLKYIHSANVLHRDLKPSNLLLNTTCDLKICDFGLARVADPDHDHTGFLTEYVATRWYRAPEIMLNSKGYTKSIDIWSVGCILAEMLSNRPIFPGKHYLDQLNHILGILGSPSQEDLNCIINLKARNYLLSLPHKNKVPWNRLFPNADSKALDLLDKMLTFNPHKRIEVEQALAHPYLEQYYDPSDEPIAEAPFKFDMELDDLPKEKLKELIFEETARFQPGYRS",
# MK06_HUMAN
"MAEKFESLMNIHGFDLGSRYMDLKPLGCGGNGLVFSAVDNDCDKRVAIKKIVLTDPQSVKHALREIKIIRRLDHDNIVKVFEILGPSGSQLTDDVGSLTELNSVYIVQEYMETDLANVLEQGPLLEEHARLFMYQLLRGLKYIHSANVLHRDLKPANLFINTEDLVLKIGDFGLARIMDPHYSHKGHLSEGLVTKWYRSPRLLLSPNNYTKAIDMWAAGCIFAEMLTGKTLFAGAHELEQMQLILESIPVVHEEDRQELLSVIPVYIRNDMTEPHKPLTQLLPGISREALDFLEQILTFSPMDRLTAEEALSHPYMSIYSFPMDEPISSHPFHIEDEVDDILLMDETHSHIYNWERYHDCQFSEHDWPVHNNFDIDEVQLDPRALSDVTDEEEVQVDPRKYLDGDREKYLEDPAFDTNYSTEPCWQYSDHHENKYCDLECSHTCNYKTRSSSYLDNLVWRESEVNHYYEPKLIIDLSNWKEQSKEKSDKKGKSKCERNGLVKAQIALEEASQQLAGKEREKNQGFDFDSFIAGTIQLSSQHEPTDVVDKLNDLNSSVSQLELKSLISKSVSQEKQEKGMANLAQLEALYQSSWDSQFVSGGEDCFFINQFCEVRKDEQVEKENTYTSYLDKFFSRKEDTEMLETEPVEDGKLGERGHEEGFLNNSGEFLFNKQLESIGIPQFHSPVGSPLKSIQATLTPSAMKSSPQIPHQTYSSILKHLN",
# MK04_HUMAN
"MAEKGDCIASVYGYDLGGRFVDFQPLGFGVNGLVLSAVDSRACRKVAVKKIALSDARSMKHALREIKIIRRLDHDNIVKVYEVLGPKGTDLQGELFKFSVAYIVQEYMETDLARLLEQGTLAEEHAKLFMYQLLRGLKYIHSANVLHRDLKPANIFISTEDLVLKIGDFGLARIVDQHYSHKGYLSEGLVTKWYRSPRLLLSPNNYTKAIDMWAAGCILAEMLTGRMLFAGAHELEQMQLILETIPVIREEDKDELLRVMPSFVSSTWEVKRPLRKLLPEVNSEAIDFLEKILTFNPMDRLTAEMGLQHPYMSPYSCPEDEPTSQHPFRIEDEIDDIVLMAANQSQLSNWDTCSSRYPVSLSSDLEWRPDRCQDASEVQRDPRAGSAPLAEDVQVDPRKDSHSSSERFLEQSHSSMERAFEADYGRSCDYKVGSPSYLDKLLWRDNKPHHYSEPKLILDLSHWKQAAGAPPTATGLADTGAREDEPASLFLEIAQWVKSTQGGPEHASPPADDPERRLSASPPGRPAPVDGGASPQFDLDVFISRALKLCTKPEDLPDNKLGDLNGACIPEHPGDLVQTEAFSKERW",
# FAK1_HUMAN
"MAAAYLDPNLNHTPNSSTKTHLGTGMERSPGAMERVLKVFHYFESNSEPTTWASIIRHGDATDVRGIIQKIVDSHKVKHVACYGFRLSHLRSEEVHWLHVDMGVSSVREKYELAHPPEEWKYELRIRYLPKGFLNQFTEDKPTLNFFYQQVKSDYMLEIADQVDQEIALKLGCLEIRRSYWEMRGNALEKKSNYEVLEKDVGLKRFFPKSLLDSVKAKTLRKLIQQTFRQFANLNREESILKFFEILSPVYRFDKECFKCALGSSWIISVELAIGPEEGISYLTDKGCNPTHLADFTQVQTIQYSNSEDKDRKGMLQLKIAGAPEPLTVTAPSLTIAENMADLIDGYCRLVNGTSQSFIIRPQKEGERALPSIPKLANSEKQGMRTHAVSVSETDDYAEIIDEEDTYTMPSTRDYEIQRERIELGRCIGEGQFGDVHQGIYMSPENPALAVAIKTCKNCTSDSVREKFLQEALTMRQFDHPHIVKLIGVITENPVWIIMELCTLGELRSFLQVRKYSLDLASLILYAYQLSTALAYLESKRFVHRDIAARNVLVSSNDCVKLGDFGLSRYMEDSTYYKASKGKLPIKWMAPESINFRRFTSASDVWMFGVCMWEILMHGVKPFQGVKNNDVIGRIENGERLPMPPNCPPTLYSLMTKCWAYDPSRRPRFTELKAQLSTILEEEKAQQEERMRMESRRQATVSWDSGGSDEAPPKPSRPGYPSPRSSEGFYPSPQHMVQTNHYQVSGYPGSHGITAMAGSIYPGQASLLDQTDSWNHRPQEIAMWQPNVEDSTVLDLRGIGQVLPTHLMEERLIRQQQEMEEDQRWLEKEERFLKPDVRLSRGSIDREDGSLQGPIGNQHIYQPVGKPDPAAPPKKPPRPGAPGHLGSLASLSSPADSYNEGVKLQPQEISPPPTANLDRSNDKVYENVTGLVKAVIEMSSKIQPAPPEEYVPMVKEVGLALRTLLATVDETIPLLPASTHREIEMAQKLLNSDLGELINKMKLAQQYVMTSLQQEYKKQMLTAAHALAVDAKNLLDVIDQARLKMLGQTRPH",
# FER_HUMAN
"MGFGSDLKNSHEAVLKLQDWELRLLETVKKFMALRIKSDKEYASTLQNLCNQVDKESTVQMNYVSNVSKSWLLMIQQTEQLSRIMKTHAEDLNSGPLHRLTMMIKDKQQVKKSYIGVHQQIEAEMIKVTKTELEKLKCSYRQLIKEMNSAKEKYKEALAKGKETEKAKERYDKATMKLHMLHNQYVLALKGAQLHQNQYYDITLPLLLDSLQKMQEEMIKALKGIFDEYSQITSLVTEEIVNVHKEIQMSVEQIDPSTEYNNFIDVHRTTAAKEQEIEFDTSLLEENENLQANEIMWNNLTAESLQVMLKTLAEELMQTQQMLLNKEEAVLELEKRIEESSETCEKKSDIVLLLSQKQALEELKQSVQQLRCTEAKFSAQKELLEQKVQENDGKEPPPVVNYEEDARSVTSMERKERLSKFESIRHSIAGIIRSPKSALGSSALSDMISISEKPLAEQDWYHGAIPRIEAQELLKKQGDFLVRESHGKPGEYVLSVYSDGQRRHFIIQYVDNMYRFEGTGFSNIPQLIDHHYTTKQVITKKSGVVLLNPIPKDKKWILSHEDVILGELLGKGNFGEVYKGTLKDKTSVAVKTCKEDLPQELKIKFLQEAKILKQYDHPNIVKLIGVCTQRQPVYIIMELVSGGDFLTFLRRKKDELKLKQLVKFSLDAAAGMLYLESKNCIHRDLAARNCLVGENNVLKISDFGMSRQEDGGVYSSSGLKQIPIKWTAPEALNYGRYSSESDVWSFGILLWETFSLGVCPYPGMTNQQAREQVERGYRMSAPQHCPEDISKIMMKCWDYKPENRPKFSELQKELTIIKRKLT",
# FES_HUMAN
"MGFSSELCSPQGHGVLQQMQEAELRLLEGMRKWMAQRVKSDREYAGLLHHMSLQDSGGQSRAISPDSPISQSWAEITSQTEGLSRLLRQHAEDLNSGPLSKLSLLIRERQQLRKTYSEQWQQLQQELTKTHSQDIEKLKSQYRALARDSAQAKRKYQEASKDKDRDKAKDKYVRSLWKLFAHHNRYVLGVRAAQLHHQHHHQLLLPGLLRSLQDLHEEMACILKEILQEYLEISSLVQDEVVAIHREMAAAAARIQPEAEYQGFLRQYGSAPDVPPCVTFDESLLEEGEPLEPGELQLNELTVESVQHTLTSVTDELAVATEMVFRRQEMVTQLQQELRNEEENTHPRERVQLLGKRQVLQEALQGLQVALCSQAKLQAQQELLQTKLEHLGPGEPPPVLLLQDDRHSTSSSEQEREGGRTPTLEILKSHISGIFRPKFSLPPPLQLIPEVQKPLHEQLWYHGAIPRAEVAELLVHSGDFLVRESQGKQEYVLSVLWDGLPRHFIIQSLDNLYRLEGEGFPSIPLLIDHLLSTQQPLTKKSGVVLHRAVPKDKWVLNHEDLVLGEQIGRGNFGEVFSGRLRADNTLVAVKSCRETLPPDLKAKFLQEARILKQYSHPNIVRLIGVCTQKQPIYIVMELVQGGDFLTFLRTEGARLRVKTLLQMVGDAAAGMEYLESKCCIHRDLAARNCLVTEKNVLKISDFGMSREEADGVYAASGGLRQVPVKWTAPEALNYGRYSSESDVWSFGILLWETFSLGASPYPNLSNQQTREFVEKGGRLPCPELCPDAVFRLMEQCWAYEPGQRPSFSTIYQELQSIRKRHR",
# FGFR1_HUMAN
"MWSWKCLLFWAVLVTATLCTARPSPTLPEQAQPWGAPVEVESFLVHPGDLLQLRCRLRDDVQSINWLRDGVQLAESNRTRITGEEVEVQDSVPADSGLYACVTSSPSGSDTTYFSVNVSDALPSSEDDDDDDDSSSEEKETDNTKPNRMPVAPYWTSPEKMEKKLHAVPAAKTVKFKCPSSGTPNPTLRWLKNGKEFKPDHRIGGYKVRYATWSIIMDSVVPSDKGNYTCIVENEYGSINHTYQLDVVERSPHRPILQAGLPANKTVALGSNVEFMCKVYSDPQPHIQWLKHIEVNGSKIGPDNLPYVQILKTAGVNTTDKEMEVLHLRNVSFEDAGEYTCLAGNSIGLSHHSAWLTVLEALEERPAVMTSPLYLEIIIYCTGAFLISCMVGSVIVYKMKSGTKKSDFHSQMAVHKLAKSIPLRRQVTVSADSSASMNSGVLLVRPSRLSSSGTPMLAGVSEYELPEDPRWELPRDRLVLGKPLGEGCFGQVVLAEAIGLDKDKPNRVTKVAVKMLKSDATEKDLSDLISEMEMMKMIGKHKNIINLLGACTQDGPLYVIVEYASKGNLREYLQARRPPGLEYCYNPSHNPEEQLSSKDLVSCAYQVARGMEYLASKKCIHRDLAARNVLVTEDNVMKIADFGLARDIHHIDYYKKTTNGRLPVKWMAPEALFDRIYTHQSDVWSFGVLLWEIFTLGGSPYPGVPVEELFKLLKEGHRMDKPSNCTNELYMMMRDCWHAVPSQRPTFKQLVEDLDRIVALTSNQEYLDLSMPLDQYSPSFPDTRSSTCSSGEDSVFSHEPLPEEPCLPRHPAQLANGGLKRR",
# FGFR2_HUMAN
"MVSWGRFICLVVVTMATLSLARPSFSLVEDTTLEPEEPPTKYQISQPEVYVAAPGESLEVRCLLKDAAVISWTKDGVHLGPNNRTVLIGEYLQIKGATPRDSGLYACTASRTVDSETWYFMVNVTDAISSGDDEDDTDGAEDFVSENSNNKRAPYWTNTEKMEKRLHAVPAANTVKFRCPAGGNPMPTMRWLKNGKEFKQEHRIGGYKVRNQHWSLIMESVVPSDKGNYTCVVENEYGSINHTYHLDVVERSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVEKNGSKYGPDGLPYLKVLKAAGVNTTDKEIEVLYIRNVTFEDAGEYTCLAGNSIGISFHSAWLTVLPAPGREKEITASPDYLEIAIYCIGVFLIACMVVTVILCRMKNTTKKPDFSSQPAVHKLTKRIPLRRQVTVSAESSSSMNSNTPLVRITTRLSSTADTPMLAGVSEYELPEDPKWEFPRDKLTLGKPLGEGCFGQVVMAEAVGIDKDKPKEAVTVAVKMLKDDATEKDLSDLVSEMEMMKMIGKHKNIINLLGACTQDGPLYVIVEYASKGNLREYLRARRPPGMEYSYDINRVPEEQMTFKDLVSCTYQLARGMEYLASQKCIHRDLAARNVLVTENNVMKIADFGLARDINNIDYYKKTTNGRLPVKWMAPEALFDRVYTHQSDVWSFGVLMWEIFTLGGSPYPGIPVEELFKLLKEGHRMDKPANCTNELYMMMRDCWHAVPSQRPTFKQLVEDLDRILTLTTNEEYLDLSQPLEQYSPSYPDTRSSCSSGDDSVFSPDPMPYEPCLPQYPHINGSVKT",
# FGFR3_HUMAN
"MGAPACALALCVAVAIVAGASSESLGTEQRVVGRAAEVPGPEPGQQEQLVFGSGDAVELSCPPPGGGPMGPTVWVKDGTGLVPSERVLVGPQRLQVLNASHEDSGAYSCRQRLTQRVLCHFSVRVTDAPSSGDDEDGEDEAEDTGVDTGAPYWTRPERMDKKLLAVPAANTVRFRCPAAGNPTPSISWLKNGREFRGEHRIGGIKLRHQQWSLVMESVVPSDRGNYTCVVENKFGSIRQTYTLDVLERSPHRPILQAGLPANQTAVLGSDVEFHCKVYSDAQPHIQWLKHVEVNGSKVGPDGTPYVTVLKTAGANTTDKELEVLSLHNVTFEDAGEYTCLAGNSIGFSHHSAWLVVLPAEEELVEADEAGSVYAGILSYGVGFFLFILVVAAVTLCRLRSPPKKGLGSPTVHKISRFPLKRQVSLESNASMSSNTPLVRIARLSSGEGPTLANVSELELPADPKWELSRARLTLGKPLGEGCFGQVVMAEAIGIDKDRAAKPVTVAVKMLKDDATDKDLSDLVSEMEMMKMIGKHKNIINLLGACTQGGPLYVLVEYAAKGNLREFLRARRPPGLDYSFDTCKPPEEQLTFKDLVSCAYQVARGMEYLASQKCIHRDLAARNVLVTEDNVMKIADFGLARDVHNLDYYKKTTNGRLPVKWMAPEALFDRVYTHQSDVWSFGVLLWEIFTLGGSPYPGIPVEELFKLLKEGHRMDKPANCTHDLYMIMRECWHAAPSQRPTFKQLVEDLDRVLTVTSTDEYLDLSAPFEQYSPGGQDTPSSSSSGDDSVFAHDLLPPAPPSSGGSRT",
# FGFR4_HUMAN
"MRLLLALLGVLLSVPGPPVLSLEASEEVELEPCLAPSLEQQEQELTVALGQPVRLCCGRAERGGHWYKEGSRLAPAGRVRGWRGRLEIASFLPEDAGRYLCLARGSMIVLQNLTLITGDSLTSSNDDEDPKSHRDPSNRHSYPQQAPYWTHPQRMEKKLHAVPAGNTVKFRCPAAGNPTPTIRWLKDGQAFHGENRIGGIRLRHQHWSLVMESVVPSDRGTYTCLVENAVGSIRYNYLLDVLERSPHRPILQAGLPANTTAVVGSDVELLCKVYSDAQPHIQWLKHIVINGSSFGADGFPYVQVLKTADINSSEVEVLYLRNVSAEDAGEYTCLAGNSIGLSYQSAWLTVLPEEDPTWTAAAPEARYTDIILYASGSLALAVLLLLAGLYRGQALHGRHPRPPATVQKLSRFPLARQFSLESGSSGKSSSSLVRGVRLSSSGPALLAGLVSLDLPLDPLWEFPRDRLVLGKPLGEGCFGQVVRAEAFGMDPARPDQASTVAVKMLKDNASDKDLADLVSEMEVMKLIGRHKNIINLLGVCTQEGPLYVIVECAAKGNLREFLRARRPPGPDLSPDGPRSSEGPLSFPVLVSCAYQVARGMQYLESRKCIHRDLAARNVLVTEDNVMKIADFGLARGVHHIDYYKKTSNGRLPVKWMAPEALFDRVYTHQSDVWSFGILLWEIFTLGGSPYPGIPVEELFSLLREGHRMDRPPHCPPELYGLMRECWHAAPSQRPTFKQLVEALDKVLLAVSEEYLDLRLTFGPYSPSGGDASSTCSSSDSVFSHDPLPLGSSSFPFGSGVQT",
# FGR_HUMAN
"MGCVFCKKLEPVATAKEDAGLEGDFRSYGAADHYGPDPTKARPASSFAHIPNYSNFSSQAINPGFLDSGTIRGVSGIGVTLFIALYDYEARTEDDLTFTKGEKFHILNNTEGDWWEARSLSSGKTGCIPSNYVAPVDSIQAEEWYFGKIGRKDAERQLLSPGNPQGAFLIRESETTKGAYSLSIRDWDQTRGDHVKHYKIRKLDMGGYYITTRVQFNSVQELVQHYMEVNDGLCNLLIAPCTIMKPQTLGLAKDAWEISRSSITLERRLGTGCFGDVWLGTWNGSTKVAVKTLKPGTMSPKAFLEEAQVMKLLRHDKLVQLYAVVSEEPIYIVTEFMCHGSLLDFLKNPEGQDLRLPQLVDMAAQVAEGMAYMERMNYIHRDLRAANILVGERLACKIADFGLARLIKDDEYNPCQGSKFPIKWTAPEAALFGRFTIKSDVWSFGILLTELITKGRIPYPGMNKREVLEQVEQGYHMPCPPGCPASLYEAMEQTWRLDPEERPTFEYLQSFLEDYFTSAEPQYQPGDQT",
# FLT3_HUMAN
"MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSSVGKSSSYPMVSESPEDLGCALRPQSSGTVYEAAAVEVDVSASITLQVLVDAPGNISCLWVFKHSSLNCQPHFDLQNRGVVSMVILKMTETQAGEYLLFIQSEATNYTILFTVSIRNTLLYTLRRPYFRKMENQDALVCISESVPEPIVEWVLCDSQGESCKEESPAVVKKEEKVLHELFGTDIRCCARNELGRECTRLFTIDLNQTPQTTLPQLFLKVGEPLWIRCKAVHVNHGFGLTWELENKALEEGNYFEMSTYSTNRTMIRILFAFVSSVARNDTGYYTCSSSKHPSQSALVTIVEKGFINATNSSEDYEIDQYEEFCFSVRFKAYPQIRCTWTFSRKSFPCEQKGLDNGYSISKFCNHKHQPGEYIFHAENDDAQFTKMFTLNIRRKPQVLAEASASQASCFSDGYPLPSWTWKKCSDKSPNCTEEITEGVWNRKANRKVFGQWVSSSTLNMSEAIKGFLVKCCAYNSLGTSCETILLNSPGPFPFIQDNISFYATIGVCLLFIVVLTLLICHKYKKQFRYESQLQMVQVTGSSDNEYFYVDFREYEYDLKWEFPRENLEFGKVLGSGAFGKVMNATAYGISKTGVSIQVAVKMLKEKADSSEREALMSELKMMTQLGSHENIVNLLGACTLSGPIYLIFEYCCYGDLLNYLRSKREKFHRTWTEIFKEHNFSFYPTFQSHPNSSMPGSREVQIHPDSDQISGLHGNSFHSEDEIEYENQKRLEEEEDLNVLTFEDLLCFAYQVAKGMEFLEFKSCVHRDLAARNVLVTHGKVVKICDFGLARDIMSDSNYVVRGNARLPVKWMAPESLFEGIYTIKSDVWSYGILLWEIFSLGVNPYPGIPVDANFYKLIQNGFKMDQPFYATEEIYIIMQSCWAFDSRKRPSFPNLTSFLGCQLADAEEAMYQNVDGRVSECPHTYQNRRPFSREMDLGLLSPQAQVEDS",
# VGFR1_HUMAN
"MVSYWDTGVLLCALLSCLLLTGSSSGSKLKDPELSLKGTQHIMQAGQTLHLQCRGEAAHKWSLPEMVSKESERLSITKSACGRNGKQFCSTLTLNTAQANHTGFYSCKYLAVPTSKKKETESAIYIFISDTGRPFVEMYSEIPEIIHMTEGRELVIPCRVTSPNITVTLKKFPLDTLIPDGKRIIWDSRKGFIISNATYKEIGLLTCEATVNGHLYKTNYLTHRQTNTIIDVQISTPRPVKLLRGHTLVLNCTATTPLNTRVQMTWSYPDEKNKRASVRRRIDQSNSHANIFYSVLTIDKMQNKDKGLYTCRVRSGPSFKSVNTSVHIYDKAFITVKHRKQQVLETVAGKRSYRLSMKVKAFPSPEVVWLKDGLPATEKSARYLTRGYSLIIKDVTEEDAGNYTILLSIKQSNVFKNLTATLIVNVKPQIYEKAVSSFPDPALYPLGSRQILTCTAYGIPQPTIKWFWHPCNHNHSEARCDFCSNNEESFILDADSNMGNRIESITQRMAIIEGKNKMASTLVVADSRISGIYICIASNKVGTVGRNISFYITDVPNGFHVNLEKMPTEGEDLKLSCTVNKFLYRDVTWILLRTVNNRTMHYSISKQKMAITKEHSITLNLTIMNVSLQDSGTYACRARNVYTGEEILQKKEITIRDQEAPYLLRNLSDHTVAISSSTTLDCHANGVPEPQITWFKNNHKIQQEPGIILGPGSSTLFIERVTEEDEGVYHCKATNQKGSVESSAYLTVQGTSDKSNLELITLTCTCVAATLFWLLLTLFIRKMKRSSSEIKTDYLSIIMDPDEVPLDEQCERLPYDASKWEFARERLKLGKSLGRGAFGKVVQASAFGIKKSPTCRTVAVKMLKEGATASEYKALMTELKILTHIGHHLNVVNLLGACTKQGGPLMVIVEYCKYGNLSNYLKSKRDLFFLNKDAALHMEPKKEKMEPGLEQGKKPRLDSVTSSESFASSGFQEDKSLSDVEEEEDSDGFYKEPITMEDLISYSFQVARGMEFLSSRKCIHRDLAARNILLSENNVVKICDFGLARDIYKNPDYVRKGDTRLPLKWMAPESIFDKIYSTKSDVWSYGVLLWEIFSLGGSPYPGVQMDEDFCSRLREGMRMRAPEYSTPEIYQIMLDCWHRDPKERPRFAELVEKLGDLLQANVQQDGKDYIPINAILTGNSGFTYSTPAFSEDFFKESISAPKFNSGSSDDVRYVNAFKFMSLERIKTFEELLPNATSMFDDYQGDSSTLLASPMLKRFTWTDSKPKASLKIDLRVTSKSKESGLSDVSRPSFCHSSCGHVSEGKRRFTYDHAELERKIACCSPPPDYNSVVLYSTPPI",
# VGFR3_HUMAN
"MQRGAALCLRLWLCLGLLDGLVSGYSMTPPTLNITEESHVIDTGDSLSISCRGQHPLEWAWPGAQEAPATGDKDSEDTGVVRDCEGTDARPYCKVLLLHEVHANDTGSYVCYYKYIKARIEGTTAASSYVFVRDFEQPFINKPDTLLVNRKDAMWVPCLVSIPGLNVTLRSQSSVLWPDGQEVVWDDRRGMLVSTPLLHDALYLQCETTWGDQDFLSNPFLVHITGNELYDIQLLPRKSLELLVGEKLVLNCTVWAEFNSGVTFDWDYPGKQAERGKWVPERRSQQTHTELSSILTIHNVSQHDLGSYVCKANNGIQRFRESTEVIVHENPFISVEWLKGPILEATAGDELVKLPVKLAAYPPPEFQWYKDGKALSGRHSPHALVLKEVTEASTGTYTLALWNSAAGLRRNISLELVVNVPPQIHEKEASSPSIYSRHSRQALTCTAYGVPLPLSIQWHWRPWTPCKMFAQRSLRRRQQQDLMPQCRDWRAVTTQDAVNPIESLDTWTEFVEGKNKTVSKLVIQNANVSAMYKCVVSNKVGQDERLIYFYVTTIPDGFTIESKPSEELLEGQPVLLSCQADSYKYEHLRWYRLNLSTLHDAHGNPLLLDCKNVHLFATPLAASLEEVAPGARHATLSLSIPRVAPEHEGHYVCEVQDRRSHDKHCHKKYLSVQALEAPRLTQNLTDLLVNVSDSLEMQCLVAGAHAPSIVWYKDERLLEEKSGVDLADSNQKLSIQRVREEDAGRYLCSVCNAKGCVNSSASVAVEGSEDKGSMEIVILVGTGVIAVFFWVLLLLIFCNMRRPAHADIKTGYLSIIMDPGEVPLEEQCEYLSYDASQWEFPRERLHLGRVLGYGAFGKVVEASAFGIHKGSSCDTVAVKMLKEGATASEHRALMSELKILIHIGNHLNVVNLLGACTKPQGPLMVIVEFCKYGNLSNFLRAKRDAFSPCAEKSPEQRGRFRAMVELARLDRRRPGSSDRVLFARFSKTEGGARRASPDQEAEDLWLSPLTMEDLVCYSFQVARGMEFLASRKCIHRDLAARNILLSESDVVKICDFGLARDIYKDPDYVRKGSARLPLKWMAPESIFDKVYTTQSDVWSFGVLLWEIFSLGASPYPGVQINEEFCQRLRDGTRMRAPELATPAIRRIMLNCWSGDPKARPAFSELVEILGDLLQGRGLQEEEEVCMAPRSSQSSEEGSFSQVSTMALHIAQADAEDSPPSLQRHSLAARYYNWVSFPGCLARGAETRGSSRMKTFEEFPMTPTTYKGSVDNQTDSGMVLASEEFEQIESRHRQESGFSCKGPGQNVAVTRAHPDSQGRRRRPERGARGGQVFYNSEYGELSEPSEEDHCSPSARVTFFTDNSY",
# MTOR_HUMAN
"MLGTGPAAATTAATTSSNVSVLQQFASGLKSRNEETRAKAAKELQHYVTMELREMSQEESTRFYDQLNHHIFELVSSSDANERKGGILAIASLIGVEGGNATRIGRFANYLRNLLPSNDPVVMEMASKAIGRLAMAGDTFTAEYVEFEVKRALEWLGADRNEGRRHAAVLVLRELAISVPTFFFQQVQPFFDNIFVAVWDPKQAIREGAVAALRACLILTTQREPKEMQKPQWYRHTFEEAEKGFDETLAKEKGMNRDDRIHGALLILNELVRISSMEGERLREEMEEITQQQLVHDKYCKDLMGFGTKPRHITPFTSFQAVQPQQSNALVGLLGYSSHQGLMGFGTSPSPAKSTLVESRCCRDLMEEKFDQVCQWVLKCRNSKNSLIQMTILNLLPRLAAFRPSAFTDTQYLQDTMNHVLSCVKKEKERTAAFQALGLLSVAVRSEFKVYLPRVLDIIRAALPPKDFAHKRQKAMQVDATVFTCISMLARAMGPGIQQDIKELLEPMLAVGLSPALTAVLYDLSRQIPQLKKDIQDGLLKMLSLVLMHKPLRHPGMPKGLAHQLASPGLTTLPEASDVGSITLALRTLGSFEFEGHSLTQFVRHCADHFLNSEHKEIRMEAARTCSRLLTPSIHLISGHAHVVSQTAVQVVADVLSKLLVVGITDPDPDIRYCVLASLDERFDAHLAQAENLQALFVALNDQVFEIRELAICTVGRLSSMNPAFVMPFLRKMLIQILTELEHSGIGRIKEQSARMLGHLVSNAPRLIRPYMEPILKALILKLKDPDPDPNPGVINNVLATIGELAQVSGLEMRKWVDELFIIIMDMLQDSSLLAKRQVALWTLGQLVASTGYVVEPYRKYPTLLEVLLNFLKTEQNQGTRREAIRVLGLLGALDPYKHKVNIGMIDQSRDASAVSLSESKSSQDSSDYSTSEMLVNMGNLPLDEFYPAVSMVALMRIFRDQSLSHHHTMVVQAITFIFKSLGLKCVQFLPQVMPTFLNVIRVCDGAIREFLFQQLGMLVSFVKSHIRPYMDEIVTLMREFWVMNTSIQSTIILLIEQIVVALGGEFKLYLPQLIPHMLRVFMHDNSPGRIVSIKLLAAIQLFGANLDDYLHLLLPPIVKLFDAPEAPLPSRKAALETVDRLTESLDFTDYASRIIHPIVRTLDQSPELRSTAMDTLSSLVFQLGKKYQIFIPMVNKVLVRHRINHQRYDVLICRIVKGYTLADEEEDPLIYQHRMLRSGQGDALASGPVETGPMKKLHVSTINLQKAWGAARRVSKDDWLEWLRRLSLELLKDSSSPSLRSCWALAQAYNPMARDLFNAAFVSCWSELNEDQQDELIRSIELALTSQDIAEVTQTLLNLAEFMEHSDKGPLPLRDDNGIVLLGERAAKCRAYAKALHYKELEFQKGPTPAILESLISINNKLQQPEAAAGVLEYAMKHFGELEIQATWYEKLHEWEDALVAYDKKMDTNKDDPELMLGRMRCLEALGEWGQLHQQCCEKWTLVNDETQAKMARMAAAAAWGLGQWDSMEEYTCMIPRDTHDGAFYRAVLALHQDLFSLAQQCIDKARDLLDAELTAMAGESYSRAYGAMVSCHMLSELEEVIQYKLVPERREIIRQIWWERLQGCQRIVEDWQKILMVRSLVVSPHEDMRTWLKYASLCGKSGRLALAHKTLVLLLGVDPSRQLDHPLPTVHPQVTYAYMKNMWKSARKIDAFQHMQHFVQTMQQQAQHAIATEDQQHKQELHKLMARCFLKLGEWQLNLQGINESTIPKVLQYYSAATEHDRSWYKAWHAWAVMNFEAVLHYKHQNQARDEKKKLRHASGANITNATTAATTAATATTTASTEGSNSESEAESTENSPTPSPLQKKVTEDLSKTLLMYTVPAVQGFFRSISLSRGNNLQDTLRVLTLWFDYGHWPDVNEALVEGVKAIQIDTWLQVIPQLIARIDTPRPLVGRLIHQLLTDIGRYHPQALIYPLTVASKSTTTARHNAANKILKNMCEHSNTLVQQAMMVSEELIRVAILWHEMWHEGLEEASRLYFGERNVKGMFEVLEPLHAMMERGPQTLKETSFNQAYGRDLMEAQEWCRKYMKSGNVKDLTQAWDLYYHVFRRISKQLPQLTSLELQYVSPKLLMCRDLELAVPGTYDPNQPIIRIQSIAPSLQVITSKQRPRKLTLMGSNGHEFVFLLKGHEDLRQDERVMQLFGLVNTLLANDPTSLRKNLSIQRYAVIPLSTNSGLIGWVPHCDTLHALIRDYREKKKILLNIEHRIMLRMAPDYDHLTLMQKVEVFEHAVNNTAGDDLAKLLWLKSPSSEVWFDRRTNYTRSLAVMSMVGYILGLGDRHPSNLMLDRLSGKILHIDFGDCFEVAMTREKFPEKIPFRLTRMLTNAMEVTGLDGNYRITCHTVMEVLREHKDSVMAVLEAFVYDPLLNWRLMDTNTKGNKRSRTRTDSYSAGQSVEILDGVELGEPAHKKTGTTVPESIHSFIGDGLVKPEALNKKAIQIINRVRDKLTGRDFSHDDTLDVPTQVELLIKQATSHENLCQCYIGWCPFW",
# FYN_HUMAN
"MGCVQCKDKEATKLTEERDGSLNQSSGYRYGTDPTPQHYPSFGVTSIPNYNNFHAAGGQGLTVFGGVNSSSHTGTLRTRGGTGVTLFVALYDYEARTEDDLSFHKGEKFQILNSSEGDWWEARSLTTGETGYIPSNYVAPVDSIQAEEWYFGKLGRKDAERQLLSFGNPRGTFLIRESETTKGAYSLSIRDWDDMKGDHVKHYKIRKLDNGGYYITTRAQFETLQQLVQHYSERAAGLCCRLVVPCHKGMPRLTDLSVKTKDVWEIPRESLQLIKRLGNGQFGEVWMGTWNGNTKVAIKTLKPGTMSPESFLEEAQIMKKLKHDKLVQLYAVVSEEPIYIVTEYMNKGSLLDFLKDGEGRALKLPNLVDMAAQVAAGMAYIERMNYIHRDLRSANILVGNGLICKIADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELVTKGRVPYPGMNNREVLEQVERGYRMPCPQDCPISLHELMIHCWKKDPEERPTFEYLQSFLEDYFTATEPQYQPGENL",
# GAK_HUMAN
"MSLLQSALDFLAGPGSLGGASGRDQSDFVGQTVELGELRLRVRRVLAEGGFAFVYEAQDVGSGREYALKRLLSNEEEKNRAIIQEVCFMKKLSGHPNIVQFCSAASIGKEESDTGQAEFLLLTELCKGQLVEFLKKMESRGPLSCDTVLKIFYQTCRAVQHMHRQKPPIIHRDLKVENLLLSNQGTIKLCDFGSATTISHYPDYSWSAQRRALVEEEITRNTTPMYRTPEIIDLYSNFPIGEKQDIWALGCILYLLCFRQHPFEDGAKLRIVNGKYSIPPHDTQYTVFHSLIRAMLQVNPEERLSIAEVVHQLQEIAAARNVNPKSPITELLEQNGGYGSATLSRGPPPPVGPAGSGYSGGLALAEYDQPYGGFLDILRGGTERLFTNLKDTSSKVIQSVANYAKGDLDISYITSRIAVMSFPAEGVESALKNNIEDVRLFLDSKHPGHYAVYNLSPRTYRPSRFHNRVSECGWAARRAPHLHTLYNICRNMHAWLRQDHKNVCVVHCMDGRAASAVAVCSFLCFCRLFSTAEAAVYMFSMKRCPPGIWPSHKRYIEYMCDMVAEEPITPHSKPILVRAVVMTPVPLFSKQRSGCRPFCEVYVGDERVASTSQEYDKMRDFKIEDGKAVIPLGVTVQGDVLIVIYHARSTLGGRLQAKMASMKMFQIQFHTGFVPRNATTVKFAKYDLDACDIQEKYPDLFQVNLEVEVEPRDRPSREAPPWENSSMRGLNPKILFSSREEQQDILSKFGKPELPRQPGSTAQYDAGAGSPEAEPTDSDSPPSSSADASRFLHTLDWQEEKEAETGAENASSKESESALMEDRDESEVSDEGGSPISSEGQEPRADPEPPGLAAGLVQQDLVFEVETPAVLPEPVPQEDGVDLLGLHSEVGAGPAVPPQACKAPSSNTDLLSCLLGPPEAASQGPPEDLLSEDPLLLASPAPPLSVQSTPRGGPPAAADPFGPLLPSSGNNSQPCSNPDLFGEFLNSDSVTVPPSFPSAHSAPPPSCSADFLHLGDLPGEPSKMTASSSNPDLLGGWAAWTETAASAVAPTPATEGPLFSPGGQPAPCGSQASWTKSQNPDPFADLGDLSSGLQGSPAGFPPGGFIPKTATTPKGSSSWQTSRPPAQGASWPPQAKPPPKACTQPRPNYASNFSVIGAREERGVRAPSFAQKPKVSENDFEDLLSNQGFSSRSDKKGPKTIAEMRKQDLAKDTDPLKLKLLDWIEGKERNIRALLSTLHTVLWDGESRWTPVGMADLVAPEQVKKHYRRAVLAVHPDKAAGQPYEQHAKMIFMELNDAWSEFENQGSRPLF",
# GRK4_HUMAN
"MELENIVANSLLLKARQGGYGKKSGRSKKWKEILTLPPVSQCSELRHSIEKDYSSLCDKQPIGRRLFRQFCDTKPTLKRHIEFLDAVAEYEVADDEDRSDCGLSILDRFFNDKLAAPLPEIPPDVVTECRLGLKEENPSKKAFEECTRVAHNYLRGEPFEEYQESSYFSQFLQWKWLERQPVTKNTFRHYRVLGKGGFGEVCACQVRATGKMYACKKLQKKRIKKRKGEAMALNEKRILEKVQSRFVVSLAYAYETKDALCLVLTIMNGGDLKFHIYNLGNPGFDEQRAVFYAAELCCGLEDLQRERIVYRDLKPENILLDDRGHIRISDLGLATEIPEGQRVRGRVGTVGYMAPEVVNNEKYTFSPDWWGLGCLIYEMIQGHSPFKKYKEKVKWEEVDQRIKNDTEEYSEKFSEDAKSICRMLLTKNPSKRLGCRGEGAAGVKQHPVFKDINFRRLEANMLEPPFCPDPHAVYCKDVLDIEQFSVVKGIYLDTADEDFYARFATGCVSIPWQNEMIESGCFKDINKSESEEALPLDLDKNIHTPVSRPNRGFFYRLFRRGGCLTMVPSEKEVEPKQC",
# GRK5_HUMAN
"MELENIVANTVLLKAREGGGGKRKGKSKKWKEILKFPHISQCEDLRRTIDRDYCSLCDKQPIGRLLFRQFCETRPGLECYIQFLDSVAEYEVTPDEKLGEKGKEIMTKYLTPKSPVFIAQVGQDLVSQTEEKLLQKPCKELFSACAQSVHEYLRGEPFHEYLDSMFFDRFLQWKWLERQPVTKNTFRQYRVLGKGGFGEVCACQVRATGKMYACKRLEKKRIKKRKGESMALNEKQILEKVNSQFVVNLAYAYETKDALCLVLTIMNGGDLKFHIYNMGNPGFEEERALFYAAEILCGLEDLHRENTVYRDLKPENILLDDYGHIRISDLGLAVKIPEGDLIRGRVGTVGYMAPEVLNNQRYGLSPDYWGLGCLIYEMIEGQSPFRGRKEKVKREEVDRRVLETEEVYSHKFSEEAKSICKMLLTKDAKQRLGCQEEGAAEVKRHPFFRNMNFKRLEAGMLDPPFVPDPRAVYCKDVLDIEQFSTVKGVNLDHTDDDFYSKFSTGSVSIPWQNEMIETECFKELNVFGPNGTLPPDLNRNHPPEPPKKGLLQRLFKRQHQNNSKSSPSSKTSFNHHINSNHVSSNSTGSS",
# GRK6_HUMAN
"MELENIVANTVLLKAREGGGGNRKGKSKKWRQMLQFPHISQCEELRLSLERDYHSLCERQPIGRLLFREFCATRPELSRCVAFLDGVAEYEVTPDDKRKACGRQLTQNFLSHTGPDLIPEVPRQLVTNCTQRLEQGPCKDLFQELTRLTHEYLSVAPFADYLDSIYFNRFLQWKWLERQPVTKNTFRQYRVLGKGGFGEVCACQVRATGKMYACKKLEKKRIKKRKGEAMALNEKQILEKVNSRFVVSLAYAYETKDALCLVLTLMNGGDLKFHIYHMGQAGFPEARAVFYAAEICCGLEDLHRERIVYRDLKPENILLDDHGHIRISDLGLAVHVPEGQTIKGRVGTVGYMAPEVVKNERYTFSPDWWALGCLLYEMIAGQSPFQQRKKKIKREEVERLVKEVPEEYSERFSPQARSLCSQLLCKDPAERLGCRGGSAREVKEHPLFKKLNFKRLGAGMLEPPFKPDPQAIYCKDVLDIEQFSTVKGVELEPTDQDFYQKFATGSVPIPWQNEMVETECFQELNVFGLDGSVPPDLDWKGQPPAPPKKGLLQRLFSRQDCCGNCSDSEEELPTRL",
# TRIB2_HUMAN
"MNIHRSTPITIARYGRSRNKTQDFEELSSIRSAEPSQSFSPNLGSPSPPETPNLSHCVSCIGKYLLLEPLEGDHVFRAVHLHSGEELVCKVFDISCYQESLAPCFCLSAHSNINQITEIILGETKAYVFFERSYGDMHSFVRTCKKLREEEAARLFYQIASAVAHCHDGGLVLRDLKLRKFIFKDEERTRVKLESLEDAYILRGDDDSLSDKHGCPAYVSPEILNTSGSYSGKAADVWSLGVMLYTMLVGRYPFHDIEPSSLFSKIRRGQFNIPETLSPKAKCLIRSILRREPSERLTSQEILDHPWFSTDFSVSNSAYGAKEVSDQLVPDVNMEENLDPFFN",
# GSK3A_HUMAN
"MSGGGPSGGGPGGSGRARTSSFAEPGGGGGGGGGGPGGSASGPGGTGGGKASVGAMGGGVGASSSGGGPGGSGGGGSGGPGAGTSFPPPGVKLGRDSGKVTTVVATLGQGPERSQEVAYTDIKVIGNGSFGVVYQARLAETRELVAIKKVLQDKRFKNRELQIMRKLDHCNIVRLRYFFYSSGEKKDELYLNLVLEYVPETVYRVARHFTKAKLTIPILYVKVYMYQLFRSLAYIHSQGVCHRDIKPQNLLVDPDTAVLKLCDFGSAKQLVRGEPNVSYICSRYYRAPELIFGATDYTSSIDVWSAGCVLAELLLGQPIFPGDSGVDQLVEIIKVLGTPTREQIREMNPNYTEFKFPQIKAHPWTKVFKSRTPPEAIALCSSLLEYTPSSRLSPLEACAHSFFDELRCLGTQLPNNRPLPPLFNFSAGELSIQPSLNAILIPPHLRSPAGTTTLTPSSQALTETPTSSDWQSTDATPTLTNSS",
# GSK3B_HUMAN
"MSGRPRTTSFAESCKPVQQPSAFGSMKVSRDKDGSKVTTVVATPGQGPDRPQEVSYTDTKVIGNGSFGVVYQAKLCDSGELVAIKKVLQDKRFKNRELQIMRKLDHCNIVRLRYFFYSSGEKKDEVYLNLVLDYVPETVYRVARHYSRAKQTLPVIYVKLYMYQLFRSLAYIHSFGICHRDIKPQNLLLDPDTAVLKLCDFGSAKQLVRGEPNVSYICSRYYRAPELIFGATDYTSSIDVWSAGCVLAELLLGQPIFPGDSGVDQLVEIIKVLGTPTREQIREMNPNYTEFKFPQIKAHPWTKVFRPRTPPEAIALCSRLLEYTPTARLTPLEACAHSFFDELRDPNVKLPNGRDTPALFNFTTQELSSNPPLATILIPPHARIQAAASTPTNATAASDANTGDRGQTNNAASASASNST",
# HCK_HUMAN
"MGGRSSCEDPGCPRDEERAPRMGCMKSKFLQVGGNTFSKTETSASPHCPVYVPDPTSTIKPGPNSHNSNTPGIREAGSEDIIVVALYDYEAIHHEDLSFQKGDQMVVLEESGEWWKARSLATRKEGYIPSNYVARVDSLETEEWFFKGISRKDAERQLLAPGNMLGSFMIRDSETTKGSYSLSVRDYDPRQGDTVKHYKIRTLDNGGFYISPRSTFSTLQELVDHYKKGNDGLCQKLSVPCMSSKPQKPWEKDAWEIPRESLKLEKKLGAGQFGEVWMATYNKHTKVAVKTMKPGSMSVEAFLAEANVMKTLQHDKLVKLHAVVTKEPIYIITEFMAKGSLLDFLKSDEGSKQPLPKLIDFSAQIAEGMAFIEQRNYIHRDLRAANILVSASLVCKIADFGLARVIEDNEYTAREGAKFPIKWTAPEAINFGSFTIKSDVWSFGILLMEIVTYGRIPYPGMSNPEVIRALERGYRMPRPENCPEELYNIMMRCWKNRPEERPTFEYIQSVLDDFYTATESQYQQQP",
# ERBB2_HUMAN
"MELAALCRWGLLLALLPPGAASTQVCTGTDMKLRLPASPETHLDMLRHLYQGCQVVQGNLELTYLPTNASLSFLQDIQEVQGYVLIAHNQVRQVPLQRLRIVRGTQLFEDNYALAVLDNGDPLNNTTPVTGASPGGLRELQLRSLTEILKGGVLIQRNPQLCYQDTILWKDIFHKNNQLALTLIDTNRSRACHPCSPMCKGSRCWGESSEDCQSLTRTVCAGGCARCKGPLPTDCCHEQCAAGCTGPKHSDCLACLHFNHSGICELHCPALVTYNTDTFESMPNPEGRYTFGASCVTACPYNYLSTDVGSCTLVCPLHNQEVTAEDGTQRCEKCSKPCARVCYGLGMEHLREVRAVTSANIQEFAGCKKIFGSLAFLPESFDGDPASNTAPLQPEQLQVFETLEEITGYLYISAWPDSLPDLSVFQNLQVIRGRILHNGAYSLTLQGLGISWLGLRSLRELGSGLALIHHNTHLCFVHTVPWDQLFRNPHQALLHTANRPEDECVGEGLACHQLCARGHCWGPGPTQCVNCSQFLRGQECVEECRVLQGLPREYVNARHCLPCHPECQPQNGSVTCFGPEADQCVACAHYKDPPFCVARCPSGVKPDLSYMPIWKFPDEEGACQPCPINCTHSCVDLDDKGCPAEQRASPLTSIISAVVGILLVVVLGVVFGILIKRRQQKIRKYTMRRLLQETELVEPLTPSGAMPNQAQMRILKETELRKVKVLGSGAFGTVYKGIWIPDGENVKIPVAIKVLRENTSPKANKEILDEAYVMAGVGSPYVSRLLGICLTSTVQLVTQLMPYGCLLDHVRENRGRLGSQDLLNWCMQIAKGMSYLEDVRLVHRDLAARNVLVKSPNHVKITDFGLARLLDIDETEYHADGGKVPIKWMALESILRRRFTHQSDVWSYGVTVWELMTFGAKPYDGIPAREIPDLLEKGERLPQPPICTIDVYMIMVKCWMIDSECRPRFRELVSEFSRMARDPQRFVVIQNEDLGPASPLDSTFYRSLLEDDDMGDLVDAEEYLVPQQGFFCPDPAPGAGGMVHHRHRSSSTRSGGGDLTLGLEPSEEEAPRSPLAPSEGAGSDVFDGDLGMGAAKGLQSLPTHDPSPLQRYSEDPTVPLPSETDGYVAPLTCSPQPEYVNQPDVRPQPPSPREGPLPAARPAGATLERPKTLSPGKNGVVKDVFAFGGAVENPEYLTPQGGAAPQPHPPPAFSPAFDNLYYWDQDPPERGAPPSTFKGTPTAENPEYLGLDVPV",
# ERBB3_HUMAN
"MRANDALQVLGLLFSLARGSEVGNSQAVCPGTLNGLSVTGDAENQYQTLYKLYERCEVVMGNLEIVLTGHNADLSFLQWIREVTGYVLVAMNEFSTLPLPNLRVVRGTQVYDGKFAIFVMLNYNTNSSHALRQLRLTQLTEILSGGVYIEKNDKLCHMDTIDWRDIVRDRDAEIVVKDNGRSCPPCHEVCKGRCWGPGSEDCQTLTKTICAPQCNGHCFGPNPNQCCHDECAGGCSGPQDTDCFACRHFNDSGACVPRCPQPLVYNKLTFQLEPNPHTKYQYGGVCVASCPHNFVVDQTSCVRACPPDKMEVDKNGLKMCEPCGGLCPKACEGTGSGSRFQTVDSSNIDGFVNCTKILGNLDFLITGLNGDPWHKIPALDPEKLNVFRTVREITGYLNIQSWPPHMHNFSVFSNLTTIGGRSLYNRGFSLLIMKNLNVTSLGFRSLKEISAGRIYISANRQLCYHHSLNWTKVLRGPTEERLDIKHNRPRRDCVAEGKVCDPLCSSGGCWGPGPGQCLSCRNYSRGGVCVTHCNFLNGEPREFAHEAECFSCHPECQPMEGTATCNGSGSDTCAQCAHFRDGPHCVSSCPHGVLGAKGPIYKYPDVQNECRPCHENCTQGCKGPELQDCLGQTLVLIGKTHLTMALTVIAGLVVIFMMLGGTFLYWRGRRIQNKRAMRRYLERGESIEPLDPSEKANKVLARIFKETELRKLKVLGSGVFGTVHKGVWIPEGESIKIPVCIKVIEDKSGRQSFQAVTDHMLAIGSLDHAHIVRLLGLCPGSSLQLVTQYLPLGSLLDHVRQHRGALGPQLLLNWGVQIAKGMYYLEEHGMVHRNLAARNVLLKSPSQVQVADFGVADLLPPDDKQLLYSEAKTPIKWMALESIHFGKYTHQSDVWSYGVTVWELMTFGAEPYAGLRLAEVPDLLEKGERLAQPQICTIDVYMVMVKCWMIDENIRPTFKELANEFTRMARDPPRYLVIKRESGPGIAPGPEPHGLTNKKLEEVELEPELDLDLDLEAEEDNLATTTLGSALSLPVGTLNRPRGSQSLLSPSSGYMPMNQGNLGESCQESAVSGSSERCPRPVSLHPMPRGCLASESSEGHVTGSEAELQEKVSMCRSRSRSRSPRPRGDSAYHSQRHSLLTPVTPLSPPGLEEEDVNGYVMPDTHLKGTPSSREGTLSSVGLSSVLGTEEEDEDEEYEYMNRRRRHSPPHPPRPSSLEELGYEYMDVGSDLSASLGSTQSCPLHPVPIMPTAGTTPDEDYEYMNRQRDGGGPGGDYAAMGACPASEQGYEEMRAFQGPGHQAPHVHYARLKTLRSLEATDSAFDNPDYWHSRLFPKANAQRT",
# ERBB4_HUMAN
"MKPATGLWVWVSLLVAAGTVQPSDSQSVCAGTENKLSSLSDLEQQYRALRKYYENCEVVMGNLEITSIEHNRDLSFLRSVREVTGYVLVALNQFRYLPLENLRIIRGTKLYEDRYALAIFLNYRKDGNFGLQELGLKNLTEILNGGVYVDQNKFLCYADTIHWQDIVRNPWPSNLTLVSTNGSSGCGRCHKSCTGRCWGPTENHCQTLTRTVCAEQCDGRCYGPYVSDCCHRECAGGCSGPKDTDCFACMNFNDSGACVTQCPQTFVYNPTTFQLEHNFNAKYTYGAFCVKKCPHNFVVDSSSCVRACPSSKMEVEENGIKMCKPCTDICPKACDGIGTGSLMSAQTVDSSNIDKFINCTKINGNLIFLVTGIHGDPYNAIEAIDPEKLNVFRTVREITGFLNIQSWPPNMTDFSVFSNLVTIGGRVLYSGLSLLILKQQGITSLQFQSLKEISAGNIYITDNSNLCYYHTINWTTLFSTINQRIVIRDNRKAENCTAEGMVCNHLCSSDGCWGPGPDQCLSCRRFSRGRICIESCNLYDGEFREFENGSICVECDPQCEKMEDGLLTCHGPGPDNCTKCSHFKDGPNCVEKCPDGLQGANSFIFKYADPDRECHPCHPNCTQGCNGPTSHDCIYYPWTGHSTLPQHARTPLIAAGVIGGLFILVIVGLTFAVYVRRKSIKKKRALRRFLETELVEPLTPSGTAPNQAQLRILKETELKRVKVLGSGAFGTVYKGIWVPEGETVKIPVAIKILNETTGPKANVEFMDEALIMASMDHPHLVRLLGVCLSPTIQLVTQLMPHGCLLEYVHEHKDNIGSQLLLNWCVQIAKGMMYLEERRLVHRDLAARNVLVKSPNHVKITDFGLARLLEGDEKEYNADGGKMPIKWMALECIHYRKFTHQSDVWSYGVTIWELMTFGGKPYDGIPTREIPDLLEKGERLPQPPICTIDVYMVMVKCWMIDADSRPKFKELAAEFSRMARDPQRYLVIQGDDRMKLPSPNDSKFFQNLLDEEDLEDMMDAEEYLVPQAFNIPPPIYTSRARIDSNRSEIGHSPPPAYTPMSGNQFVYRDGGFAAEQGVSVPYRAPTSTIPEAPVAQGATAEIFDDSCCNGTLRKPVAPHVQEDSSTQRYSADPTVFAPERSPRGELDEEGYMTPMRDKPKQEYLNPVEENPFVSRRKNGDLQALDNPEYHNASNGPPKAEDEYVNEPLYLNTFANTLGKAEYLKNNILSMPEKAKKAFDNPDYWNHSLPPRSTLQHPDYLQEYSTKYFYKQNGRIRPIVAENPEYLSEFSLKPGTVLPPPPYRHRNTVV",
# HIPK1_HUMAN
"MASQLQVFSPPSVSSSAFCSAKKLKIEPSGWDVSGQSSNDKYYTHSKTLPATQGQANSSHQVANFNIPAYDQGLLLPAPAVEHIVVTAADSSGSAATSTFQSSQTLTHRSNVSLLEPYQKCGLKRKSEEVDSNGSVQIIEEHPPLMLQNRTVVGAAATTTTVTTKSSSSSGEGDYQLVQHEILCSMTNSYEVLEFLGRGTFGQVAKCWKRSTKEIVAIKILKNHPSYARQGQIEVSILSRLSSENADEYNFVRSYECFQHKNHTCLVFEMLEQNLYDFLKQNKFSPLPLKYIRPILQQVATALMKLKSLGLIHADLKPENIMLVDPVRQPYRVKVIDFGSASHVSKAVCSTYLQSRYYRAPEIILGLPFCEAIDMWSLGCVIAELFLGWPLYPGASEYDQIRYISQTQGLPAEYLLSAGTKTTRFFNRDPNLGYPLWRLKTPEEHELETGIKSKEARKYIFNCLDDMAQVNMSTDLEGTDMLAEKADRREYIDLLKKMLTIDADKRITPLKTLNHQFVTMTHLLDFPHSNHVKSCFQNMEICKRRVHMYDTVSQIKSPFTTHVAPNTSTNLTMSFSNQLNTVHNQASVLASSSTAAAATLSLANSDVSLLNYQSALYPSSAAPVPGVAQQGVSLQPGTTQICTQTDPFQQTFIVCPPAFQTGLQATTKHSGFPVRMDNAVPIVPQAPAAQPLQIQSGVLTQGSCTPLMVATLHPQVATITPQYAVPFTLSCAAGRPALVEQTAAVLQAWPGGTQQILLPSTWQQLPGVALHNSVQPTAMIPEAMGSGQQLADWRNAHSHGNQYSTIMQQPSLLTNHVTLATAQPLNVGVAHVVRQQQSSSLPSKKNKQSAPVSSKSSLDVLPSQVYSLVGSSPLRTTSSYNSLVPVQDQHQPIIIPDTPSPPVSVITIRSDTDEEEDNKYKPSSSGLKPRSNVISYVTVNDSPDSDSSLSSPYSTDTLSALRGNSGSVLEGPGRVVADGTGTRTIIVPPLKTQLGDCTVATQASGLLSNKTKPVASVSGQSSGCCITPTGYRAQRGGTSAAQPLNLSQNQQSSAAPTSQERSSNPAPRRQQAFVAPLSQAPYTFQHGSPLHSTGHPHLAPAPAHLPSQAHLYTYAAPTSAAALGSTSSIAHLFSPQGSSRHAAAYTTHPSTLVHQVPVSVGPSLLTSASVAPAQYQHQFATQSYIGSSRGSTIYTGYPLSPTKISQYSYL",
# M4K1_HUMAN
"MDVVDPDIFNRDPRDHYDLLQRLGGGTYGEVFKARDKVSGDLVALKMVKMEPDDDVSTLQKEILILKTCRHANIVAYHGSYLWLQKLWICMEFCGAGSLQDIYQVTGSLSELQISYVCREVLQGLAYLHSQKKIHRDIKGANILINDAGEVRLADFGISAQIGATLARRLSFIGTPYWMAPEVAAVALKGGYNELCDIWSLGITAIELAELQPPLFDVHPLRVLFLMTKSGYQPPRLKEKGKWSAAFHNFIKVTLTKSPKKRPSATKMLSHQLVSQPGLNRGLILDLLDKLKNPGKGPSIGDIEDEEPELPPAIPRRIRSTHRSSSLGIPDADCCRRHMEFRKLRGMETRPPANTARLQPPRDLRSSSPRKQLSESSDDDYDDVDIPTPAEDTPPPLPPKPKFRSPSDEGPGSMGDDGQLSPGVLVRCASGPPPNSPRPGPPPSTSSPHLTAHSEPSLWNPPSRELDKPPLLPPKKEKMKRKGCALLVKLFNGCPLRIHSTAAWTHPSTKDQHLLLGAEEGIFILNRNDQEATLEMLFPSRTTWVYSINNVLMSLSGKTPHLYSHSILGLLERKETRAGNPIAHISPHRLLARKNMVSTKIQDTKGCRACCVAEGASSGGPFLCGALETSVVLLQWYQPMNKFLLVRQVLFPLPTPLSVFALLTGPGSELPAVCIGVSPGRPGKSVLFHTVRFGALSCWLGEMSTEHRGPVQVTQVEEDMVMVLMDGSVKLVTPEGSPVRGLRTPEIPMTEAVEAVAMVGGQLQAFWKHGVQVWALGSDQLLQELRDPTLTFRLLGSPRLECSGTISPHCNLLLPGSSNSPASASRVAGITGL",
# GUC2C_HUMAN
"MKTLLLDLALWSLLFQPGWLSFSSQVSQNCHNGSYEISVLMMGNSAFAEPLKNLEDAVNEGLEIVRGRLQNAGLNVTVNATFMYSDGLIHNSGDCRSSTCEGLDLLRKISNAQRMGCVLIGPSCTYSTFQMYLDTELSYPMISAGSFGLSCDYKETLTRLMSPARKLMYFLVNFWKTNDLPFKTYSWSTSYVYKNGTETEDCFWYLNALEASVSYFSHELGFKVVLRQDKEFQDILMDHNRKSNVIIMCGGPEFLYKLKGDRAVAEDIVIILVDLFNDQYFEDNVTAPDYMKNVLVLTLSPGNSLLNSSFSRNLSPTKRDFALAYLNGILLFGHMLKIFLENGENITTPKFAHAFRNLTFEGYDGPVTLDDWGDVDSTMVLLYTSVDTKKYKVLLTYDTHVNKTYPVDMSPTFTWKNSKLPNDITGRGPQILMIAVFTLTGAVVLLLLVALLMLRKYRKDYELRQKKWSHIPPENIFPLETNETNHVSLKIDDDKRRDTIQRLRQCKYDKKRVILKDLKHNDGNFTEKQKIELNKLLQIDYYNLTKFYGTVKLDTMIFGVIEYCERGSLREVLNDTISYPDGTFMDWEFKISVLYDIAKGMSYLHSSKTEVHGRLKSTNCVVDSRMVVKITDFGCNSILPPKKDLWTAPEHLRQANISQKGDVYSYGIIAQEIILRKETFYTLSCRDRNEKIFRVENSNGMKPFRPDLFLETAEEKELEVYLLVKNCWEEDPEKRPDFKKIETTLAKIFGLFHDQKNESYMDTLIRRLQLYSRNLEHLVEERTQLYKAERDRADRLNFMLLPRLVVKSLKEKGFVEPELYEEVTIYFSDIVGFTTICKYSTPMEVVDMLNDIYKSFDHIVDHHDVYKVETIGDAYMVASGLPKRNGNRHAIDIAKMALEILSFMGTFELEHLPGLPIWIRIGVHSGPCAAGVVGIKMPRYCLFGDTVNTASRMESTGLPLRIHVSGSTIAILKRTECQFLYEVRGETYLKGRGNETTYWLTGMKDQKFNLPTPPTVENQQRLQAEFSDMIANSLQKRQAAGIRSQKPRRVASYKKGTLEYLQLNTTDKESTYF",
# IGF1R_HUMAN
"MKSGSGGGSPTSLWGLLFLSAALSLWPTSGEICGPGIDIRNDYQQLKRLENCTVIEGYLHILLISKAEDYRSYRFPKLTVITEYLLLFRVAGLESLGDLFPNLTVIRGWKLFYNYALVIFEMTNLKDIGLYNLRNITRGAIRIEKNADLCYLSTVDWSLILDAVSNNYIVGNKPPKECGDLCPGTMEEKPMCEKTTINNEYNYRCWTTNRCQKMCPSTCGKRACTENNECCHPECLGSCSAPDNDTACVACRHYYYAGVCVPACPPNTYRFEGWRCVDRDFCANILSAESSDSEGFVIHDGECMQECPSGFIRNGSQSMYCIPCEGPCPKVCEEEKKTKTIDSVTSAQMLQGCTIFKGNLLINIRRGNNIASELENFMGLIEVVTGYVKIRHSHALVSLSFLKNLRLILGEEQLEGNYSFYVLDNQNLQQLWDWDHRNLTIKAGKMYFAFNPKLCVSEIYRMEEVTGTKGRQSKGDINTRNNGERASCESDVLHFTSTTTSKNRIIITWHRYRPPDYRDLISFTVYYKEAPFKNVTEYDGQDACGSNSWNMVDVDLPPNKDVEPGILLHGLKPWTQYAVYVKAVTLTMVENDHIRGAKSEILYIRTNASVPSIPLDVLSASNSSSQLIVKWNPPSLPNGNLSYYIVRWQRQPQDGYLYRHNYCSKDKIPIRKYADGTIDIEEVTENPKTEVCGGEKGPCCACPKTEAEKQAEKEEAEYRKVFENFLHNSIFVPRPERKRRDVMQVANTTMSSRSRNTTAADTYNITDPEELETEYPFFESRVDNKERTVISNLRPFTLYRIDIHSCNHEAEKLGCSASNFVFARTMPAEGADDIPGPVTWEPRPENSIFLKWPEPENPNGLILMYEIKYGSQVEDQRECVSRQEYRKYGGAKLNRLNPGNYTARIQATSLSGNGSWTDPVFFYVQAKTGYENFIHLIIALPVAVLLIVGGLVIMLYVFHRKRNNSRLGNGVLYASVNPEYFSAADVYVPDEWEVAREKITMSRELGQGSFGMVYEGVAKGVVKDEPETRVAIKTVNEAASMRERIEFLNEASVMKEFNCHHVVRLLGVVSQGQPTLVIMELMTRGDLKSYLRSLRPEMENNPVLAPPSLSKMIQMAGEIADGMAYLNANKFVHRDLAARNCMVAEDFTVKIGDFGMTRDIYETDYYRKGGKGLLPVRWMSPESLKDGVFTTYSDVWSFGVVLWEIATLAEQPYQGLSNEQVLRFVMEGGLLDKPDNCPDMLFELMRMCWQYNPKMRPSFLEIISSIKEEMEPGFREVSFYYSEENKLPEPEELDLEPENMESVPLDPSASSSSLPLPDRHSGHKAENGPGPGVLVLRASFDERQPYAHMNGGRKNERALPLPQSSTC",
# IKKA_HUMAN
"MERPPGLRPGAGGPWEMRERLGTGGFGNVCLYQHRELDLKIAIKSCRLELSTKNRERWCHEIQIMKKLNHANVVKACDVPEELNILIHDVPLLAMEYCSGGDLRKLLNKPENCCGLKESQILSLLSDIGSGIRYLHENKIIHRDLKPENIVLQDVGGKIIHKIIDLGYAKDVDQGSLCTSFVGTLQYLAPELFENKPYTATVDYWSFGTMVFECIAGYRPFLHHLQPFTWHEKIKKKDPKCIFACEEMSGEVRFSSHLPQPNSLCSLVVEPMENWLQLMLNWDPQQRGGPVDLTLKQPRCFVLMDHILNLKIVHILNMTSAKIISFLLPPDESLHSLQSRIERETGINTGSQELLSETGISLDPRKPASQCVLDGVRGCDSYMVYLFDKSKTVYEGPFASRSLSDCVNYIVQDSKIQLPIIQLRKVWAEAVHYVSGLKEDYSRLFQGQRAAMLSLLRYNANLTKMKNTLISASQQLKAKLEFFHKSIQLDLERYSEQMTYGISSEKMLKAWKEMEEKAIHYAEVGVIGYLEDQIMSLHAEIMELQKSPYGRRQGDLMESLEQRAIDLYKQLKHRPSDHSYSDSTEMVKIIVHTVQSQDRVLKELFGHLSKLLGCKQKIIDLLPKVEVALSNIKEADNTVMFMQGKRQKEIWHLLKIACTQSSARSLVGSSLEGAVTPQTSAWLPPTSAEHDHSLSCVVTPQDGETSAQMIEENLNCLGHLSTIIHEANEEQGNSMMNLDWSWLTE",
# IKKB_HUMAN
"MSWSPSLTTQTCGAWEMKERLGTGGFGNVIRWHNQETGEQIAIKQCRQELSPRNRERWCLEIQIMRRLTHPNVVAARDVPEGMQNLAPNDLPLLAMEYCQGGDLRKYLNQFENCCGLREGAILTLLSDIASALRYLHENRIIHRDLKPENIVLQQGEQRLIHKIIDLGYAKELDQGSLCTSFVGTLQYLAPELLEQQKYTVTVDYWSFGTLAFECITGFRPFLPNWQPVQWHSKVRQKSEVDIVVSEDLNGTVKFSSSLPYPNNLNSVLAERLEKWLQLMLMWHPRQRGTDPTYGPNGCFKALDDILNLKLVHILNMVTGTIHTYPVTEDESLQSLKARIQQDTGIPEEDQELLQEAGLALIPDKPATQCISDGKLNEGHTLDMDLVFLFDNSKITYETQISPRPQPESVSCILQEPKRNLAFFQLRKVWGQVWHSIQTLKEDCNRLQQGQRAAMMNLLRNNSCLSKMKNSMASMSQQLKAKLDFFKTSIQIDLEKYSEQTEFGITSDKLLLAWREMEQAVELCGRENEVKLLVERMMALQTDIVDLQRSPMGRKQGGTLDDLEEQARELYRRLREKPRDQRTEGDSQEMVRLLLQAIQSFEKKVRVIYTQLSKTVVCKQKALELLPKVEEVVSLMNEDEKTVVRLQEKRQKELWNLLKIACSKVRGPVSGSPDSMNASRLSQPGQLMSQPSTASNSLPEPAKKSEELVAEAHNLCTLLENAIQDTVREQDQSFTALDWSWLQTEEEEHSCLEQAS",
# ILK_HUMAN
"MDDIFTQCREGNAVAVRLWLDNTENDLNQGDDHGFSPLHWACREGRSAVVEMLIMRGARINVMNRGDDTPLHLAASHGHRDIVQKLLQYKADINAVNEHGNVPLHYACFWGQDQVAEDLVANGALVSICNKYGEMPVDKAKAPLRELLRERAEKMGQNLNRIPYKDTFWKGTTRTRPRNGTLNKHSGIDFKQLNFLTKLNENHSGELWKGRWQGNDIVVKVLKVRDWSTRKSRDFNEECPRLRIFSHPNVLPVLGACQSPPAPHPTLITHWMPYGSLYNVLHEGTNFVVDQSQAVKFALDMARGMAFLHTLEPLIPRHALNSRSVMIDEDMTARISMADVKFSFQCPGRMYAPAWVAPEALQKKPEDTNRRSADMWSFAVLLWELVTREVPFADLSNMEIGMKVALEGLRPTIPPGISPHVCKLMKICMNEDPAKRPKFDMIVPILEKMQDK",
# INSR_HUMAN
"MATGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLTRLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFPNLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRILDSVEDNYIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTICKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHFQDWRCVNFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCHLLEGEKTIDSVTSAQELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRRSYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNLRQLWDWSKHNLTITQGKLFFHYNPKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDQASCENELLKFSYIRTSFDKILLRWEPYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHPGWLMRGLKPWTQYAIFVKTLVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSSSQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCLKGLKLPSRTWSPPFESEDSQKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAEDPRPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGLRHFTGYRIELQACNQDTPEERCSVAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMWQEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRGLSPGNYSVRIRATSLAGNGSWTEPTYFYVTDYLDVPSNIAKIIIGPLIFVFLFSVVIGSIYLFLRKRQPDGPLGPLYASSNPEYLSASDVFPCSVYVPDEWEVSREKITLLRELGQGSFGMVYEGNARDIIKGEAETRVAVKTVNESASLRERIEFLNEASVMKGFTCHHVVRLLGVVSKGQPTLVVMELMAHGDLKSYLRSLRPEAENNPGRPPPTLQEMIQMAAEIADGMAYLNAKKFVHRDLAARNCMVAHDFTVKIGDFGMTRDIYETDYYRKGGKGLLPVRWMAPESLKDGVFTTSSDMWSFGVVLWEITSLAEQPYQGLSNEQVLKFVMDGGYLDQPDNCPERVTDLMRMCWQFNPKMRPTFLEIVNLLKDDLHPSFPEVSFFHSEENKAPESEELEMEFEDMENVPLDRSSHCQREEAGGRDGGSSLGFKRSYEEHIPYTHMNGGKKNGRILTLPRSNPS",
# IRAK1_HUMAN
"MAGGPGPGEPAAPGAQHFLYEVPPWVMCRFYKVMDALEPADWCQFAALIVRDQTELRLCERSGQRTASVLWPWINRNARVADLVHILTHLQLLRARDIITAWHPPAPLPSPGTTAPRPSSIPAPAEAEAWSPRKLPSSASTFLSPAFPGSQTHSGPELGLVPSPASLWPPPPSPAPSSTKPGPESSVSLLQGARPFPFCWPLCEISRGTHNFSEELKIGEGGFGCVYRAVMRNTVYAVKRLKENADLEWTAVKQSFLTEVEQLSRFRHPNIVDFAGYCAQNGFYCLVYGFLPNGSLEDRLHCQTQACPPLSWPQRLDILLGTARAIQFLHQDSPSLIHGDIKSSNVLLDERLTPKLGDFGLARFSRFAGSSPSQSSMVARTQTVRGTLAYLPEEYIKTGRLAVDTDTFSFGVVVLETLAGQRAVKTHGARTKYLKDLVEEEAEEAGVALRSTQSTLQAGLAADAWAAPIAMQIYKKHLDPRPGPCPPELGLGLGQLACCCLHRRAKRRPPMTQVYERLEKLQAVVAGVPGHSEAASCIPPSPQENSYVSSTGRAHSGAAPWQPLAAPSGASAQAAEQLQRGPNQPVESDESLGGLSAALRSWHLTPSCPLDPAPLREAGCPQGDTAGESSWGSGPGSRPTAVEGLALGSSASSSSEPPQIIINPARQKMVQKLALYEDGALDSLQLLSSSSLPGLGLEQDRQGPEESDEFQS",
# IRAK2_HUMAN
"MACYIYQLPSWVLDDLCRNMDALSEWDWMEFASYVITDLTQLRKIKSMERVQGVSITRELLWWWGMRQATVQQLVDLLCRLELYRAAQIILNWKPAPEIRCPIPAFPDSVKPEKPLAASVRKAEDEQEEGQPVRMATFPGPGSSPARAHQPAFLQPPEEDAPHSLRSDLPTSSDSKDFSTSIPKQEKLLSLAGDSLFWSEADVVQATDDFNQNRKISQGTFADVYRGHRHGKPFVFKKLRETACSSPGSIERFFQAELQICLRCCHPNVLPVLGFCAARQFHSFIYPYMANGSLQDRLQGQGGSDPLPWPQRVSICSGLLCAVEYLHGLEIIHSNVKSSNVLLDQNLTPKLAHPMAHLCPVNKRSKYTMMKTHLLRTSAAYLPEDFIRVGQLTKRVDIFSCGIVLAEVLTGIPAMDNNRSPVYLKDLLLSDIPSSTASLCSRKTGVENVMAKEICQKYLEKGAGRLPEDCAEALATAACLCLRRRNTSLQEVCGSVAAVEERLRGRETLLPWSGLSEGTGSSSNTPEETDDVDNSSLDASSSMSVAPWAGAATPLLPTENGEGRLRVIVGREADSSSEACVGLEPPQDVTETSWQIEINEAKRKLMENILLYKEEKVDSIELFGP",
# IRAK3_HUMAN
"MAGNCGARGALSAHTLLFDLPPALLGELCAVLDSCDGALGWRGLAERLSSSWLDVRHIEKYVDQGKSGTRELLWSWAQKNKTIGDLLQVLQEMGHRRAIHLITNYGAVLSPSEKSYQEGGFPNILFKETANVTVDNVLIPEHNEKGILLKSSISFQNIIEGTRNFHKDFLIGEGEIFEVYRVEIQNLTYAVKLFKQEKKMQCKKHWKRFLSELEVLLLFHHPNILELAAYFTETEKFCLIYPYMRNGTLFDRLQCVGDTAPLPWHIRIGILIGISKAIHYLHNVQPCSVICGSISSANILLDDQFQPKLTDFAMAHFRSHLEHQSCTINMTSSSSKHLWYMPEEYIRQGKLSIKTDVYSFGIVIMEVLTGCRVVLDDPKHIQLRDLLRELMEKRGLDSCLSFLDKKVPPCPRNFSAKLFCLAGRCAATRAKLRPSMDEVLNTLESTQASLYFAEDPPTSLKSFRCPSPLFLENVPSIPVEDDESQNNNLLPSDEGLRIDRMTQKTPFECSQSEVMFLSLDKKPESKRNEEACNMPSSSCEESWFPKYIVPSQDLRPYKVNIDPSSEAPGHSCRSRPVESSCSSKFSWDEYEQYKKE",
# ERN1_HUMAN
"MPARRLLLLLTLLLPGLGIFGSTSTVTLPETLLFVSTLDGSLHAVSKRTGSIKWTLKEDPVLQVPTHVEEPAFLPDPNDGSLYTLGSKNNEGLTKLPFTIPELVQASPCRSSDGILYMGKKQDIWYVIDLLTGEKQQTLSSAFADSLCPSTSLLYLGRTEYTITMYDTKTRELRWNATYFDYAASLPEDDVDYKMSHFVSNGDGLVVTVDSESGDVLWIQNYASPVVAFYVWQREGLRKVMHINVAVETLRYLTFMSGEVGRITKWKYPFPKETEAKSKLTPTLYVGKYSTSLYASPSMVHEGVAVVPRGSTLPLLEGPQTDGVTIGDKGECVITPSTDVKFDPGLKSKNKLNYLRNYWLLIGHHETPLSASTKMLERFPNNLPKHRENVIPADSEKKSFEEVINLVDQTSENAPTTVSRDVEEKPAHAPARPEAPVDSMLKDMATIILSTFLLIGWVAFIITYPLSMHQQQQLQHQQFQKELEKIQLLQQQQQQLPFHPPGDTAQDGELLDTSGPYSESSGTSSPSTSPRASNHSLCSGSSASKAGSSPSLEQDDGDEETSVVIVGKISFCPKDVLGHGAEGTIVYRGMFDNRDVAVKRILPECFSFADREVQLLRESDEHPNVIRYFCTEKDRQFQYIAIELCAATLQEYVEQKDFAHLGLEPITLLQQTTSGLAHLHSLNIVHRDLKPHNILISMPNAHGKIKAMISDFGLCKKLAVGRHSFSRRSGVPGTEGWIAPEMLSEDCKENPTYTVDIFSAGCVFYYVISEGSHPFGKSLQRQANILLGACSLDCLHPEKHEDVIARELIEKMIAMDPQKRPSAKHVLKHPFFWSLEKQLQFFQDVSDRIEKESLDGPIVKQLERGGRAVVKMDWRENITVPLQTDLRKFRTYKGGSVRDLLRAMRNKKHHYRELPAEVRETLGSLPDDFVCYFTSRFPHLLAHTYRAMELCSHERLFQPYYFHEPPEPQPPVTPDAL",
# INSRR_HUMAN
"MAVPSLWPWGACLPVIFLSLGFGLDTVEVCPSLDIRSEVAELRQLENCSVVEGHLQILLMFTATGEDFRGLSFPRLTQVTDYLLLFRVYGLESLRDLFPNLAVIRGTRLFLGYALVIFEMPHLRDVALPALGAVLRGAVRVEKNQELCHLSTIDWGLLQPAPGANHIVGNKLGEECADVCPGVLGAAGEPCAKTTFSGHTDYRCWTSSHCQRVCPCPHGMACTARGECCHTECLGGCSQPEDPRACVACRHLYFQGACLWACPPGTYQYESWRCVTAERCASLHSVPGRASTFGIHQGSCLAQCPSGFTRNSSSIFCHKCEGLCPKECKVGTKTIDSIQAAQDLVGCTHVEGSLILNLRQGYNLEPQLQHSLGLVETITGFLKIKHSFALVSLGFFKNLKLIRGDAMVDGNYTLYVLDNQNLQQLGSWVAAGLTIPVGKIYFAFNPRLCLEHIYRLEEVTGTRGRQNKAEINPRTNGDRAACQTRTLRFVSNVTEADRILLRWERYEPLEARDLLSFIVYYKESPFQNATEHVGPDACGTQSWNLLDVELPLSRTQEPGVTLASLKPWTQYAVFVRAITLTTEEDSPHQGAQSPIVYLRTLPAAPTVPQDVISTSNSSSHLLVRWKPPTQRNGNLTYYLVLWQRLAEDGDLYLNDYCHRGLRLPTSNNDPRFDGEDGDPEAEMESDCCPCQHPPPGQVLPPLEAQEASFQKKFENFLHNAITIPISPWKVTSINKSPQRDSGRHRRAAGPLRLGGNSSDFEIQEDKVPRERAVLSGLRHFTEYRIDIHACNHAAHTVGCSAATFVFARTMPHREADGIPGKVAWEASSKNSVLLRWLEPPDPNGLILKYEIKYRRLGEEATVLCVSRLRYAKFGGVHLALLPPGNYSARVRATSLAGNGSWTDSVAFYILGPEEEDAGGLHVLLTATPVGLTLLIVLAALGFFYGKKRNRTLYASVNPEYFSASDMYVPDEWEVPREQISIIRELGQGSFGMVYEGLARGLEAGEESTPVALKTVNELASPRECIEFLKEASVMKAFKCHHVVRLLGVVSQGQPTLVIMELMTRGDLKSHLRSLRPEAENNPGLPQPALGEMIQMAGEIADGMAYLAANKFVHRDLAARNCMVSQDFTVKIGDFGMTRDVYETDYYRKGGKGLLPVRWMAPESLKDGIFTTHSDVWSFGVVLWEIVTLAEQPYQGLSNEQVLKFVMDGGVLEELEGCPLQLQELMSRCWQPNPRLRPSFTHILDSIQEELRPSFRLLSFYYSPECRGARGSLPTTDAEPDSSPTPRDCSPQNGGPGH",
# ITK_HUMAN
"MNNFILLEEQLIKKSQQKRRTSPSNFKVRFFVLTKASLAYFEDRHGKKRTLKGSIELSRIKCVEIVKSDISIPCHYKYPFQVVHDNYLLYVFAPDRESRQRWVLALKEETRNNNSLVPKYHPNFWMDGKWRCCSQLEKLATGCAQYDPTKNASKKPLPPTPEDNRRPLWEPEETVVIALYDYQTNDPQELALRRNEEYCLLDSSEIHWWRVQDRNGHEGYVPSSYLVEKSPNNLETYEWYNKSISRDKAEKLLLDTGKEGAFMVRDSRTAGTYTVSVFTKAVVSENNPCIKHYHIKETNDNPKRYYVAEKYVFDSIPLLINYHQHNGGGLVTRLRYPVCFGRQKAPVTAGLRYGKWVIDPSELTFVQEIGSGQFGLVHLGYWLNKDKVAIKTIREGAMSEEDFIEEAEVMMKLSHPKLVQLYGVCLEQAPICLVFEFMEHGCLSDYLRTQRGLFAAETLLGMCLDVCEGMAYLEEACVIHRDLAARNCLVGENQVIKVSDFGMTRFVLDDQYTSSTGTKFPVKWASPEVFSFSRYSSKSDVWSFGVLMWEVFSEGKIPYENRSNSEVVEDISTGFRLYKPRLASTHVYQIMNHCWKERPEDRPAFSRLLRQLAEIAESGL",
# JAK1_HUMAN
"MQYLNIKEDCNAMAFCAKMRSSKKTEVNLEAPEPGVEVIFYLSDREPLRLGSGEYTAEELCIRAAQACRISPLCHNLFALYDENTKLWYAPNRTITVDDKMSLRLHYRMRFYFTNWHGTNDNEQSVWRHSPKKQKNGYEKKKIPDATPLLDASSLEYLFAQGQYDLVKCLAPIRDPKTEQDGHDIENECLGMAVLAISHYAMMKKMQLPELPKDISYKRYIPETLNKSIRQRNLLTRMRINNVFKDFLKEFNNKTICDSSVSTHDLKVKYLATLETLTKHYGAEIFETSMLLISSENEMNWFHSNDGGNVLYYEVMVTGNLGIQWRHKPNVVSVEKEKNKLKRKKLENKHKKDEEKNKIREEWNNFSYFPEITHIVIKESVVSINKQDNKKMELKLSSHEEALSFVSLVDGYFRLTADAHHYLCTDVAPPLIVHNIQNGCHGPICTEYAINKLRQEGSEEGMYVLRWSCTDFDNILMTVTCFEKSEQVQGAQKQFKNFQIEVQKGRYSLHGSDRSFPSLGDLMSHLKKQILRTDNISFMLKRCCQPKPREISNLLVATKKAQEWQPVYPMSQLSFDRILKKDLVQGEHLGRGTRTHIYSGTLMDYKDDEGTSEEKKIKVILKVLDPSHRDISLAFFEAASMMRQVSHKHIVYLYGVCVRDVENIMVEEFVEGGPLDLFMHRKSDVLTTPWKFKVAKQLASALSYLEDKDLVHGNVCTKNLLLAREGIDSECGPFIKLSDPGIPITVLSRQECIERIPWIAPECVEDSKNLSVAADKWSFGTTLWEICYNGEIPLKDKTLIEKERFYESRCRPVTPSCKELADLMTRCMNYDPNQRPFFRAIMRDINKLEEQNPDIVSEKKPATEVDPTHFEKRFLKRIRDLGEGHFGKVELCRYDPEGDNTGEQVAVKSLKPESGGNHIADLKKEIEILRNLYHENIVKYKGICTEDGGNGIKLIMEFLPSGSLKEYLPKNKNKINLKQQLKYAVQICKGMDYLGSRQYVHRDLAARNVLVESEHQVKIGDFGLTKAIETDKEYYTVKDDRDSPVFWYAPECLMQSKFYIASDVWSFGVTLHELLTYCDSDSSPMALFLKMIGPTHGQMTVTRLVNTLKEGKRLPCPPNCPDEVYQLMRKCWEFQPSNRTSFQNLIEGFEALLK",
# JAK2_HUMAN
"MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHSLGKSEADYLTFPSGEYVAEEICIAASKACGITPVYHNMFALMSETERIWYPPNHVFHIDESTRHNVLYRIRFYFPRWYCSGSNRAYRHGISRGAEAPLLDDFVMSYLFAQWRHDFVHGWIKVPVTHETQEECLGMAVLDMMRIAKENDQTPLAIYNSISYKTFLPKCIRAKIQDYHILTRKRIRYRFRRFIQQFSQCKATARNLKLKYLINLETLQSAFYTEKFEVKEPGSGPSGEEIFATIIITGNGGIQWSRGKHKESETLTEQDLQLYCDFPNIIDVSIKQANQEGSNESRVVTIHKQDGKNLEIELSSLREALSFVSLIDGYYRLTADAHHYLCKEVAPPAVLENIQSNCHGPISMDFAISKLKKAGNQTGLYVLRCSPKDFNKYFLTFAVERENVIEYKHCLITKNENEEYNLSGTKKNFSSLKDLLNCYQMETVRSDNIIFQFTKCCPPKPKDKSNLLVFRTNGVSDVPTSPTLQRPTHMNQMVFHKIRNEDLIFNESLGQGTFTKIFKGVRREVGDYGQLHETEVLLKVLDKAHRNYSESFFEAASMMSKLSHKHLVLNYGVCVCGDENILVQEFVKFGSLDTYLKKNKNCINILWKLEVAKQLAWAMHFLEENTLIHGNVCAKNILLIREEDRKTGNPPFIKLSDPGISITVLPKDILQERIPWVPPECIENPKNLNLATDKWSFGTTLWEICSGGDKPLSALDSQRKLQFYEDRHQLPAPKWAELANLINNCMDYEPDFRPSFRAIIRDLNSLFTPDYELLTENDMLPNMRIGALGFSGAFEDRDPTQFEERHLKFLQQLGKGNFGSVEMCRYDPLQDNTGEVVAVKKLQHSTEEHLRDFEREIEILKSLQHDNIVKYKGVCYSAGRRNLKLIMEYLPYGSLRDYLQKHKERIDHIKLLQYTSQICKGMEYLGTKRYIHRDLATRNILVENENRVKIGDFGLTKVLPQDKEYYKVKEPGESPIFWYAPESLTESKFSVASDVWSFGVVLYELFTYIEKSKSPPAEFMRMIGNDKQGQMIVFHLIELLKNNGRLPRPDGCPDEIYMIMTECWNNNVNQRPSFRDLALRVDQIRDNMAG",
# JAK3_HUMAN
"MAPPSEETPLIPQRSCSLLSTEAGALHVLLPARGPGPPQRLSFSFGDHLAEDLCVQAAKASGILPVYHSLFALATEDLSCWFPPSHIFSVEDASTQVLLYRIRFYFPNWFGLEKCHRFGLRKDLASAILDLPVLEHLFAQHRSDLVSGRLPVGLSLKEQGECLSLAVLDLARMAREQAQRPGELLKTVSYKACLPPSLRDLIQGLSFVTRRRIRRTVRRALRRVAACQADRHSLMAKYIMDLERLDPAGAAETFHVGLPGALGGHDGLGLLRVAGDGGIAWTQGEQEVLQPFCDFPEIVDISIKQAPRVGPAGEHRLVTVTRTDNQILEAEFPGLPEALSFVALVDGYFRLTTDSQHFFCKEVAPPRLLEEVAEQCHGPITLDFAINKLKTGGSRPGSYVLRRSPQDFDSFLLTVCVQNPLGPDYKGCLIRRSPTGTFLLVGLSRPHSSLRELLATCWDGGLHVDGVAVTLTSCCIPRPKEKSNLIVVQRGHSPPTSSLVQPQSQYQLSQMTFHKIPADSLEWHENLGHGSFTKIYRGCRHEVVDGEARKTEVLLKVMDAKHKNCMESFLEAASLMSQVSYRHLVLLHGVCMAGDSTMVQEFVHLGAIDMYLRKRGHLVPASWKLQVVKQLAYALNYLEDKGLPHGNVSARKVLLAREGADGSPPFIKLSDPGVSPAVLSLEMLTDRIPWVAPECLREAQTLSLEADKWGFGATVWEVFSGVTMPISALDPAKKLQFYEDRQQLPAPKWTELALLIQQCMAYEPVQRPSFRAVIRDLNSLISSDYELLSDPTPGALAPRDGLWNGAQLYACQDPTIFEERHLKYISQLGKGNFGSVELCRYDPLGDNTGALVAVKQLQHSGPDQQRDFQREIQILKALHSDFIVKYRGVSYGPGRQSLRLVMEYLPSGCLRDFLQRHRARLDASRLLLYSSQICKGMEYLGSRRCVHRDLAARNILVESEAHVKIADFGLAKLLPLDKDYYVVREPGQSPIFWYAPESLSDNIFSRQSDVWSFGVVLYELFTYCDKSCSPSAEFLRMMGCERDVPALCRLLELLEEGQRLPAPPACPAEVHELMKLCWAPSPQDRPSFSALGPQLDMLWSGSRGCETHAFTAHPEGKHHSLSFS",
# MK08_HUMAN
"MSRSKRDNNFYSVEIGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDAILERNVAIKKLSRPFQNQTHAKRAYRELVLMKCVNHKNIIGLLNVFTPQKSLEEFQDVYIVMELMDANLCQVIQMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTAGTSFMMTPYVVTRYYRAPEVILGMGYKENVDLWSVGCIMGEMVCHKILFPGRDYIDQWNKVIEQLGTPCPEFMKKLQPTVRTYVENRPKYAGYSFEKLFPDVLFPADSEHNKLKASQARDLLSKMLVIDASKRISVDEALQHPYINVWYDPSEAEAPPPKIPDKQLDEREHTIEEWKELIYKEVMDLEERTKNGVIRGQPSPLGAAVINGSQHPSSSSSVNDVSSMSTDPTLASDTDSSLEAAAGPLGCCR",
# MK09_HUMAN
"MSDSKCDSQFYSVQVADSTFTVLKRYQQLKPIGSGAQGIVCAAFDTVLGINVAVKKLSRPFQNQTHAKRAYRELVLLKCVNHKNIISLLNVFTPQKTLEEFQDVYLVMELMDANLCQVIHMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTACTNFMMTPYVVTRYYRAPEVILGMGYKENVDIWSVGCIMGELVKGCVIFQGTDHIDQWNKVIEQLGTPSAEFMKKLQPTVRNYVENRPKYPGIKFEELFPDWIFPSESERDKIKTSQARDLLSKMLVIDPDKRISVDEALRHPYITVWYDPAEAEAPPPQIYDAQLEEREHAIEEWKELIYKEVMDWEERSKNGVVKDQPSDAAVSSNATPSQSSSINDISSMSTEQTLASDTDSSLDASTGPLEGCR",
# MK10_HUMAN
"MSLHFLYYCSEPTLDVKIAFCQGFDKQVDVSYIAKHYNMSKSKVDNQFYSVEVGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDAVLDRNVAIKKLSRPFQNQTHAKRAYRELVLMKCVNHKNIISLLNVFTPQKTLEEFQDVYLVMELMDANLCQVIQMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTAGTSFMMTPYVVTRYYRAPEVILGMGYKENVDIWSVGCIMGEMVRHKILFPGRDYIDQWNKVIEQLGTPCPEFMKKLQPTVRNYVENRPKYAGLTFPKLFPDSLFPADSEHNKLKASQARDLLSKMLVIDPAKRISVDDALQHPYINVWYDPAEVEAPPPQIYDKQLDEREHTIEEWKELIYKEVMNSEEKTKNGVVKGQPSPSGAAVNSSESLPPSSSVNDISSMSTDQTLASDTDSSLEASAGPLGCCR",
# M4K5_HUMAN
"MEAPLRPAADILRRNPQQDYELVQRVGSGTYGDVYKARNVHTGELAAVKIIKLEPGDDFSLIQQEIFMVKECKHCNIVAYFGSYLSREKLWICMEYCGGGSLQDIYHVTGPLSELQIAYVCRETLQGLAYLHTKGKMHRDIKGANILLTDHGDVKLADFGVAAKITATIAKRKSFIGTPYWMAPEVAAVEKNGGYNQLCDIWAVGITAIELGELQPPMFDLHPMRALFLMSKSNFQPPKLKDKTKWSSTFHNFVKIALTKNPKKRPTAERLLTHTFVAQPGLSRALAVELLDKVNNPDNHAHYTEADDDDFEPHAIIRHTIRSTNRNARAERTASEINFDKLQFEPPLRKETEARDEMGLSSDPNFMLQWNPFVDGANTGKSTSKRAIPPPLPPKPRISSYPEDNFPDEEKASTIKHCPDSESRAPQILRRQSSPSCGPVAETSSIGNGDGISKLMSENTEGSAQAPQLPRKKDKRDFPKPAINGLPPTPKVLMGACFSKVFDGCPLKINCATSWIHPDTKDQYIIFGTEDGIYTLNLNELHEATMEQLFPRKCTWLYVINNTLMSLSEGKTFQLYSHNLIALFEHAKKPGLAAHIQTHRFPDRILPRKFALTTKIPDTKGCHKCCIVRNPYTGHKYLCGALQSGIVLLQWYEPMQKFMLIKHFDFPLPSPLNVFEMLVIPEQEYPMVCVAISKGTESNQVVQFETINLNSASSWFTEIGAGSQQLDSIHVTQLERDTVLVCLDKFVKIVNLQGKLKSSKKLASELSFDFRIESVVCLQDSVLAFWKHGMQGKSFKSDEVTQEISDETRVFRLLGSDRVVVLESRPTENPTAHSNLYILAGHENSY",
# IKKE_HUMAN
"MQSTANYLWHTDDLLGQGATASVYKARNKKSGELVAVKVFNTTSYLRPREVQVREFEVLRKLNHQNIVKLFAVEETGGSRQKVLVMEYCSSGSLLSVLESPENAFGLPEDEFLVVLRCVVAGMNHLRENGIVHRDIKPGNIMRLVGEEGQSIYKLTDFGAARELDDDEKFVSVYGTEEYLHPDMYERAVLRKPQQKAFGVTVDLWSIGVTLYHAATGSLPFIPFGGPRRNKEIMYRITTEKPAGAIAGAQRRENGPLEWSYTLPITCQLSLGLQSQLVPILANILEVEQAKCWGFDQFFAETSDILQRVVVHVFSLSQAVLHHIYIHAHNTIAIFQEAVHKQTSVAPRHQEYLFEGHLCVLEPSVSAQHIAHTTASSPLTLFSTAIPKGLAFRDPALDVPKFVPKVDLQADYNTAKGVLGAGYQALRLARALLDGQELMFRGLHWVMEVLQATCRRTLEVARTSLLYLSSSLGTERFSSVAGTPEIQELKAAAELRSRLRTLAEVLSRCSQNITETQESLSSLNRELVKSRDQVHEDRSIQQIQCCLDKMNFIYKQFKKSRMRPGLGYNEEQIHKLDKVNFSHLAKRLLQVFQEECVQKYQASLVTHGKRMRVVHETRNHLRLVGCSVAACNTEAQGVQESLSKLLEELSHQLLQDRAKGAQASPPPIAPYPSPTRKDLLLHMQELCEGMKLLASDLLDNNRIIERLNRVPAPPDV",
# NUAK1_HUMAN
"MEGAAAPVAGDRPDLGLGAPGSPREAVAGATAALEPRKPHGVKRHHHKHNLKHRYELQETLGKGTYGKVKRATERFSGRVVAIKSIRKDKIKDEQDMVHIRREIEIMSSLNHPHIISIYEVFENKDKIVIIMEYASKGELYDYISERRRLSERETRHFFRQIVSAVHYCHKNGVVHRDLKLENILLDDNCNIKIADFGLSNLYQKDKFLQTFCGSPLYASPEIVNGRPYRGPEVDSWALGVLLYTLVYGTMPFDGFDHKNLIRQISSGEYREPTQPSDARGLIRWMLMVNPDRRATIEDIANHWWVNWGYKSSVCDCDALHDSESPLLARIIDWHHRSTGLQADTEAKMKGLAKPTTSEVMLERQRSLKKSKKENDFAQSGQDAVPESPSKLSSKRPKGILKKRSNSEHRSHSTGFIEGVVGPALPSTFKMEQDLCRTGVLLPSSPEAEVPGKLSPKQSATMPKKGILKKTQQRESGYYSSPERSESSELLDSNDVMGSSIPSPSPPDPARVTSHSLSCRRKGILKHSSKYSAGTMDPALVSPEMPTLESLSEPGVPAEGLSRSYSRPSSVISDDSVLSSDSFDLLDLQENRPARQRIRSCVSAENFLQIQDFEGLQNRPRPQYLKRYRNRLADSSFSLLTDMDDVTQVYKQALEICSKLN",
# MAST3_HUMAN
"MDESSLLRRRGLQKELSLPRRGRGCRSGNRKSLVVGTPSPTLSRPLSPLSVPTAGSSPLDSPRNFSAASALNFPFARRADGRRWSLASLPSSGYGTNTPSSTLSSSSSSRERLHQLPFQPTPDELHFLSKHFRSSENVLDEEGGRSPRLRPRSRSLSPGRATGTFDNEIVMMNHVYRERFPKATAQMEGRLQEFLTAYAPGARLALADGVLGFIHHQIVELARDCLAKSGENLVTSRYFLEMQEKLERLLQDAHERSDSEEVSFIVQLVRKLLIIISRPARLLECLEFDPEEFYHLLEAAEGHAREGQGIKTDLPQYIIGQLGLAKDPLEEMVPLSHLEEEQPPAPESPESRALVGQSRRKPCESDFETIKLISNGAYGAVYLVRHRDTRQRFAIKKINKQNLILRNQIQQVFVERDILTFAENPFVVSMFCSFETRRHLCMVMEYVEGGDCATLLKNMGPLPVDMARLYFAETVLALEYLHNYGIVHRDLKPDNLLITSLGHIKLTDFGLSKIGLMSMATNLYEGHIEKDAREFIDKQVCGTPEYIAPEVIFRQGYGKPVDWWAMGVVLYEFLVGCVPFFGDTPEELFGQVVSDEIMWPEGDEALPADAQDLITRLLRQSPLDRLGTGGTHEVKQHPFFLALDWAGLLRHKAEFVPQLEAEDDTSYFDTRSERYRHLGSEDDETNDEESSTEIPQFSSCSHRFSKVYSSSEFLAVQPTPTFAERSFSEDREEGWERSEVDYGRRLSADIRLRSWTSSGSSCQSSSSQPERGPSPSLLNTISLDTMPKFAFSSEDEGVGPGPAGPKRPVFILGEPDPPPAATPVMPKPSSLSADTAALSHARLRSNSIGARHSTPRPLDAGRGRRLGGPRDPAPEKSRASSSGGSGGGSGGRVPKSASVSALSLIITADDGSGGPLMSPLSPRSLSSNPSSRDSSPSRDPSPVCGSLRPPIVIHSSGKKYGFSLRAIRVYMGDSDVYTVHHVVWSVEDGSPAQEAGLRAGDLITHINGESVLGLVHMDVVELLLKSGNKISLRTTALENTSIKVGPARKNVAKGRMARRSKRSRRRETQDRRKSLFKKISKQTSVLHTSRSFSSGLHHSLSSSESLPGSPTHSLSPSPTTPCRSPAPDVPADTTASPPSASPSSSSPASPAAAGHTRPSSLHGLAAKLGPPRPKTGRRKSTSSIPPSPLACPPISAPPPRSPSPLPGHPPAPARSPRLRRGQSADKLGTGERLDGEAGRRTRGPEAELVVMRRLHLSERRDSFKKQEAVQEVSFDEPQEEATGLPTSVPQIAVEGEEAVPVALGPTGRD",
# STK36_HUMAN
"MEKYHVLEMIGEGSFGRVYKGRRKYSAQVVALKFIPKLGRSEKELRNLQREIEIMRGLRHPNIVHMLDSFETDKEVVVVTDYAEGELFQILEDDGKLPEDQVQAIAAQLVSALYYLHSHRILHRDMKPQNILLAKGGGIKLCDFGFARAMSTNTMVLTSIKGTPLYMSPELVEERPYDHTADLWSVGCILYELAVGTPPFYATSIFQLVSLILKDPVRWPSTISPCFKNFLQGLLTKDPRQRLSWPDLLYHPFIAGHVTIITEPAGPDLGTPFTSRLPPELQVLKDEQAHRLAPKGNQSRILTQAYKRMAEEAMQKKHQNTGPALEQEDKTSKVAPGTAPLPRLGATPQESSLLAGILASELKSSWAKSGTGEVPSAPRENRTTPDCERAFPEERPEVLGQRSTDVVDLENEEPDSDNEWQHLLETTEPVPIQLKAPLTLLCNPDFCQRIQSQLHEAGGQILKGILEGASHILPAFRVLSSLLSSCSDSVALYSFCREAGLPGLLLSLLRHSQESNSLQQQSWYGTFLQDLMAVIQAYFACTFNLERSQTSDSLQVFQEAANLFLDLLGKLLAQPDDSEQTLRRDSLMCFTVLCEAMDGNSRAISKAFYSSLLTTQQVVLDGLLHGLTVPQLPVHTPQGAPQVSQPLREQSEDIPGAISSALAAICTAPVGLPDCWDAKEQVCWHLANQLTEDSSQLRPSLISGLQHPILCLHLLKVLYSCCLVSEGLCRLLGQEPLALESLFMLIQGKVKVVDWEESTEVTLYFLSLLVFRLQNLPCGMEKLGSDVATLFTHSHVVSLVSAAACLLGQLGQQGVTFDLQPMEWMAAATHALSAPAEVRLTPPGSCGFYDGLLILLLQLLTEQGKASLIRDMSSSEMWTVLWHRFSMVLRLPEEASAQEGELSLSSPPSPEPDWTLISPQGMAALLSLAMATFTQEPQLCLSCLSQHGSILMSILKHLLCPSFLNQLRQAPHGSEFLPVVVLSVCQLLCFPFALDMDADLLIGVLADLRDSEVAAHLLQVCCYHLPLMQVELPISLLTRLALMDPTSLNQFVNTVSASPRTIVSFLSVALLSDQPLLTSDLLSLLAHTARVLSPSHLSFIQELLAGSDESYRPLRSLLGHPENSVRAHTYRLLGHLLQHSMALRGALQSQSGLLSLLLLGLGDKDPVVRCSASFAVGNAAYQAGPLGPALAAAVPSMTQLLGDPQAGIRRNVASALGNLGPEGLGEELLQCEVPQRLLEMACGDPQPNVKEAALIALRSLQQEPGIHQVLVSLGASEKLSLLSLGNQSLPHSSPRPASAKHCRKLIHLLRPAHSM",
# PIM3_HUMAN
"MLLSKFGSLAHLCGPGGVDHLPVKILQPAKADKESFEKAYQVGAVLGSGGFGTVYAGSRIADGLPVAVKHVVKERVTEWGSLGGATVPLEVVLLRKVGAAGGARGVIRLLDWFERPDGFLLVLERPEPAQDLFDFITERGALDEPLARRFFAQVLAAVRHCHSCGVVHRDIKDENLLVDLRSGELKLIDFGSGALLKDTVYTDFDGTRVYSPPEWIRYHRYHGRSATVWSLGVLLYDMVCGDIPFEQDEEILRGRLLFRRRVSPECQQLIRWCLSLRPSERPSLDQIAAHPWMLGADGGVPESCDLRLCTLDPDDVASTTSSSESL",
# KIT_HUMAN
"MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDLIVRVGDEIRLLCTDPGFVKWTFEILDETNENKQNEWITEKAEATNTGKYTCTNKHGLSNSIYVFVRDPAKLFLVDRSLYGKEDNDTLVRCPLTDPEVTNYSLKGCQGKPLPKDLRFIPDPKAGIMIKSVKRAYHRLCLHCSVDQEGKSVLSEKFILKVRPAFKAVPVVSVSKASYLLREGEEFTVTCTIKDVSSSVYSTWKRENSQTKLQEKYNSWHHGDFNYERQATLTISSARVNDSGVFMCYANNTFGSANVTTTLEVVDKGFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYTFLVSNSDVNAAIAFNVYVNTKPEILTYDRLVNGMLQCVAAGFPEPTIDWYFCPGTEQRCSASVLPVDVQTLNSSGPPFGKLVVQSSIDSSAFKHNGTVECKAYNDVGKTSAYFNFAFKGNNKEQIHPHTLFTPLLIGFVIVAGMMCIIVMILTYKYLQKPMYEVQWKVVEEINGNNYVYIDPTQLPYDHKWEFPRNRLSFGKTLGAGAFGKVVEATAYGLIKSDAAMTVAVKMLKPSAHLTEREALMSELKVLSYLGNHMNIVNLLGACTIGGPTLVITEYCCYGDLLNFLRRKRDSFICSKQEDHAEAALYKNLLHSKESSCSDSTNEYMDMKPGVSYVVPTKADKRRSVRIGSYIERDVTPAIMEDDELALDLEDLLSFSYQVAKGMAFLASKNCIHRDLAARNILLTHGRITKICDFGLARDIKNDSNYVVKGNARLPVKWMAPESIFNCVYTFESDVWSYGIFLWELFSLGSSPYPGMPVDSKFYKMIKEGFRMLSPEHAPAEMYDIMKTCWDADPLKRPTFKQIVQLIEKQISESTNHIYSNLANCSPNRQKPVVDHSVRINSVGSTASSSQPLLVHDDV",
# CDKL2_HUMAN
"MEKYENLGLVGEGSYGMVMKCRNKDTGRIVAIKKFLESDDDKMVKKIAMREIKLLKQLRHENLVNLLEVCKKKKRWYLVFEFVDHTILDDLELFPNGLDYQVVQKYLFQIINGIGFCHSHNIIHRDIKPENILVSQSGVVKLCDFGFARTLAAPGEVYTDYVATRWYRAPELLVGDVKYGKAVDVWAIGCLVTEMFMGEPLFPGDSDIDQLYHIMMCLGNLIPRHQELFNKNPVFAGVRLPEIKEREPLERRYPKLSEVVIDLAKKCLHIDPDKRPFCAELLHHDFFQMDGFAERFSQELQLKVQKDARNVSLSKKSQNRKKEKEKDDSLVEERKTLVVQDTNADPKIKDYKLFKIKGSKIDGEKAEKGNRASNASCLHDSRTSHNKIVPSTSLKDCSNVSVDHTRNPSVAIPPLTHNLSAVAPSINSGMGTETIPIQGYRVDEKTKKCSIPFVKPNRHSPSGIYNINVTTLVSGPPLSDDSGADLPQMEHQH",
# CDKL1_HUMAN
"MEKYEKIGKIGEGSYGVVFKCRNRDTGQIVAIKKFLESEDDPVIKKIALREIRMLKQLKHPNLVNLLEVFRRKRRLHLVFEYCDHTVLHELDRYQRGVPEHLVKSITWQTLQAVNFCHKHNCIHRDVKPENILITKHSVIKLCDFGFARLLTGPSDYYTDYVATRWYRSPELLVGDTQYGPPVDVWAIGCVFAELLSGVPLWPGKSDVDQLYLIRKTLGDLIPRHQQVFSTNQYFSGVKIPDPEDMEPLELKFPNISYPALGLLKGCLHMDPTQRLTCEQLLHHPYFENIREIEDLAKEHNKPTRKTLRKSRKHHCFTETSKLQYLPQLTGSSILPALDNKKYYCDTKKLNYRFPNI",
# KSR1_HUMAN
"MDRAALRAAAMGEKKEGGGGGDAAAAEGGAGAAASRALQQCGQLQKLIDISIGSLRGLRTKCAVSNDLTQQEIRTLEAKLVRYICKQRQCKLSVAPGERTPELNSYPRFSDWLYTFNVRPEVVQEIPRDLTLDALLEMNEAKVKETLRRCGASGDECGRLQYALTCLRKVTGLGGEHKEDSSWSSLDARRESGSGPSTDTLSAASLPWPPGSSQLGRAGNSAQGPRSISVSALPASDSPTPSFSEGLSDTCIPLHASGRLTPRALHSFITPPTTPQLRRHTKLKPPRTPPPPSRKVFQLLPSFPTLTRSKSHESQLGNRIDDVSSMRFDLSHGSPQMVRRDIGLSVTHRFSTKSWLSQVCHVCQKSMIFGVKCKHCRLKCHNKCTKEAPACRISFLPLTRLRRTESVPSDINNPVDRAAEPHFGTLPKALTKKEHPPAMNHLDSSSNPSSTTSSTPSSPAPFPTSSNPSSATTPPNPSPGQRDSRFNFPAAYFIHHRQQFIFPVPSAGHCWKCLLIAESLKENAFNISAFAHAAPLPEAADGTRLDDQPKADVLEAHEAEAEEPEAGKSEAEDDEDEVDDLPSSRRPWRGPISRKASQTSVYLQEWDIPFEQVELGEPIGQGRWGRVHRGRWHGEVAIRLLEMDGHNQDHLKLFKKEVMNYRQTRHENVVLFMGACMNPPHLAIITSFCKGRTLHSFVRDPKTSLDINKTRQIAQEIIKGMGYLHAKGIVHKDLKSKNVFYDNGKVVITDFGLFGISGVVREGRRENQLKLSHDWLCYLAPEIVREMTPGKDEDQLPFSKAADVYAFGTVWYELQARDWPLKNQAAEASIWQIGSGEGMKRVLTSVSLGKEVSEILSACWAFDLQERPSFSLLMDMLEKLPKLNRRLSHPGHFWKSADINSSKVVPRFERFGLGVLESSNPKM",
# LCK_HUMAN
"MGCGCSSHPEDDWMENIDVCENCHYPIVPLDGKGTLLIRNGSEVRDPLVTYEGSNPPASPLQDNLVIALHSYEPSHDGDLGFEKGEQLRILEQSGEWWKAQSLTTGQEGFIPFNFVAKANSLEPEPWFFKNLSRKDAERQLLAPGNTHGSFLIRESESTAGSFSLSVRDFDQNQGEVVKHYKIRNLDNGGFYISPRITFPGLHELVRHYTNASDGLCTRLSRPCQTQKPQKPWWEDEWEVPRETLKLVERLGAGQFGEVWMGYYNGHTKVAVKSLKQGSMSPDAFLAEANLMKQLQHQRLVRLYAVVTQEPIYIITEYMENGSLVDFLKTPSGIKLTINKLLDMAAQIAEGMAFIEERNYIHRDLRAANILVSDTLSCKIADFGLARLIEDNEYTAREGAKFPIKWTAPEAINYGTFTIKSDVWSFGILLTEIVTHGRIPYPGMTNPEVIQNLERGYRMVRPDNCPEELYQLMRLCWKERPEDRPTFDYLRSVLEDFFTATEGQYQPQP",
# LIMK2_HUMAN
"MSALAGEDVWRCPGCGDHIAPSQIWYRTVNETWHGSCFRCSECQDSLTNWYYEKDGKLYCPKDYWGKFGEFCHGCSLLMTGPFMVAGEFKYHPECFACMSCKVIIEDGDAYALVQHATLYCGKCHNEVVLAPMFERLSTESVQEQLPYSVTLISMPATTEGRRGFSVSVESACSNYATTVQVKEVNRMHISPNNRNAIHPGDRILEINGTPVRTLRVEEVEDAISQTSQTLQLLIEHDPVSQRLDQLRLEARLAPHMQNAGHPHALSTLDTKENLEGTLRRRSLRRSNSISKSPGPSSPKEPLLFSRDISRSESLRCSSSYSQQIFRPCDLIHGEVLGKGFFGQAIKVTHKATGKVMVMKELIRCDEETQKTFLTEVKVMRSLDHPNVLKFIGVLYKDKKLNLLTEYIEGGTLKDFLRSMDPFPWQQKVRFAKGIASGMAYLHSMCIIHRDLNSHNCLIKLDKTVVVADFGLSRLIVEERKRAPMEKATTKKRTLRKNDRKKRYTVVGNPYWMAPEMLNGKSYDETVDIFSFGIVLCEIIGQVYADPDCLPRTLDFGLNVKLFWEKFVPTDCPPAFFPLAAICCRLEPESRPAFSKLEDSFEALSLYLGELGIPLPAELEELDHTVSMQYGLTRDSPP",
# STK11_HUMAN
"MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLIGKYLMGDLLGEGSYGKVKEVLDSETLCRRAVKILKKKKLRRIPNGEANVKKEIQLLRRLRHKNVIQLVDVLYNEEKQKMYMVMEYCVCGMQEMLDSVPEKRFPVCQAHGYFCQLIDGLEYLHSQGIVHKDIKPGNLLLTTGGTLKISDLGVAEALHPFAADDTCRTSQGSPAFQPPEIANGLDTFSGFKVDIWSAGVTLYNITTGLYPFEGDNIYKLFENIGKGSYAIPGDCGPPLSDLLKGMLEYEPAKRFSIRQIRQHSWFRKKHPPAEAPVPIPPSPDTKDRWRSMTVVPYLEDLHGADEDEDLFDIEDDIIYTQDFTVPGQVPEEEASHNGQRRGLPKAVCMNGTEAAQLSTKSRAEGRAPNPARKACSASSKIRRLSACKQQ",
# LTK_HUMAN
"MGCWGQLLVWFGAAGAILCSSPGSQETFLRSSPLPLASPSPRDPKVSAPPSILEPASPLNSPGTEGSWLFSTCGASGRHGPTQTQCDGAYAGTSVVVTVGAAGQLRGVQLWRVPGPGQYLISAYGAAGGKGAKNHLSRAHGVFVSAIFSLGLGESLYILVGQQGEDACPGGSPESQLVCLGESRAVEEHAAMDGSEGVPGSRRWAGGGGGGGGATYVFRVRAGELEPLLVAAGGGGRAYLRPRDRGRTQASPEKLENRSEAPGSGGRGGAAGGGGGWTSRAPSPQAGRSLQEGAEGGQGCSEAWATLGWAAAGGFGGGGGACTAGGGGGGYRGGDASETDNLWADGEDGVSFIHPSSELFLQPLAVTENHGEVEIRRHLNCSHCPLRDCQWQAELQLAECLCPEGMELAVDNVTCMDLHKPPGPLVLMVAVVATSTLSLLMVCGVLILVKQKKWQGLQEMRLPSPELELSKLRTSAIRTAPNPYYCQVGLGPAQSWPLPPGVTEVSPANVTLLRALGHGAFGEVYEGLVIGLPGDSSPLQVAIKTLPELCSPQDELDFLMEALIISKFRHQNIVRCVGLSLRATPRLILLELMSGGDMKSFLRHSRPHLGQPSPLVMRDLLQLAQDIAQGCHYLEENHFIHRDIAARNCLLSCAGPSRVAKIGDFGMARDIYRASYYRRGDRALLPVKWMPPEAFLEGIFTSKTDSWSFGVLLWEIFSLGYMPYPGRTNQEVLDFVVGGGRMDPPRGCPGPVYRIMTQCWQHEPELRPSFASILERLQYCTQDPDVLNSLLPMELGPTPEEEGTSGLGNRSLECLRPPQPQELSPEKLKSWGGSPLGPWLSSGLKPLKSRGLQPQNLWNPTYRS",
# LYN_HUMAN
"MGCIKSKGKDSLSDDGVDLKTQPVRNTERTIYVRDPTSNKQQRPVPESQLLPGQRFQTKDPEEQGDIVVALYPYDGIHPDDLSFKKGEKMKVLEEHGEWWKAKSLLTKKEGFIPSNYVAKLNTLETEEWFFKDITRKDAERQLLAPGNSAGAFLIRESETLKGSFSLSVRDFDPVHGDVIKHYKIRSLDNGGYYISPRITFPCISDMIKHYQKQADGLCRRLEKACISPKPQKPWDKDAWEIPRESIKLVKRLGAGQFGEVWMGYYNNSTKVAVKTLKPGTMSVQAFLEEANLMKTLQHDKLVRLYAVVTREEPIYIITEYMAKGSLLDFLKSDEGGKVLLPKLIDFSAQIAEGMAYIERKNYIHRDLRAANVLVSESLMCKIADFGLARVIEDNEYTAREGAKFPIKWTAPEAINFGCFTIKSDVWSFGILLYEIVTYGKIPYPGRTNADVMTALSQGYRMPRVENCPDELYDIMKMCWKEKAEERPTFDYLQSVLDDFYTATEGQYQQQP",
# MAK_HUMAN
"MNRYTTMRQLGDGTYGSVLMGKSNESGELVAIKRMKRKFYSWDECMNLREVKSLKKLNHANVIKLKEVIRENDHLYFIFEYMKENLYQLMKDRNKLFPESVIRNIMYQILQGLAFIHKHGFFHRDMKPENLLCMGPELVKIADFGLARELRSQPPYTDYVSTRWYRAPEVLLRSSVYSSPIDVWAVGSIMAELYMLRPLFPGTSEVDEIFKICQVLGTPKKSDWPEGYQLASSMNFRFPQCVPINLKTLIPNASNEAIQLMTEMLNWDPKKRPTASQALKHPYFQVGQVLGPSSNHLESKQSLNKQLQPLESKPSLVEVEPKPLPDIIDQVVGQPQPKTSQQPLQPIQPPQNLSVQQPPKQQSQEKPPQTLFPSIVKNMPTKPNGTLSHKSGRRRWGQTIFKSGDSWEELEDYDFGASHSKKPSMGVFKEKRKKDSPFRLPEPVPSGSNHSTGENKSLPAVTSLKSDSELSTAPTSKQYYLKQSRYLPGVNPKKVSLIASGKEINPHTWSNQLFPKSLGPVGAELAFKRSNAGNLGSYATYNQSGYIPSFLKKEVQSAGQRIHLAPLNATASEYTWNTKTGRGQFSGRTYNPTAKNLNIVNRAQPIPSVHGRTDWVAKYGGHR",
# MAPK2_HUMAN
"MLSNSQGQSPPVPFPAPAPPPQPPTPALPHPPAQPPPPPPQQFPQFHVKSGLQIKKNAIIDDYKVTSQVLGLGINGKVLQIFNKRTQEKFALKMLQDCPKARREVELHWRASQCPHIVRIVDVYENLYAGRKCLLIVMECLDGGELFSRIQDRGDQAFTEREASEIMKSIGEAIQYLHSINIAHRDVKPENLLYTSKRPNAILKLTDFGFAKETTSHNSLTTPCYTPYYVAPEVLGPEKYDKSCDMWSLGVIMYILLCGYPPFYSNHGLAISPGMKTRIRMGQYEFPNPEWSEVSEEVKMLIRNLLKTEPTQRMTITEFMNHPWIMQSTKVPQTPLHTSRVLKEDKERWEDVKEEMTSALATMRVDYEQIKIKKIEDASNPLLLKRRKKARALEAAALAH",
# MAPK3_HUMAN
"MDGETAEEQGGPVPPPVAPGGPGLGGAPGGRREPKKYAVTDDYQLSKQVLGLGVNGKVLECFHRRTGQKCALKLLYDSPKARQEVDHHWQASGGPHIVCILDVYENMHHGKRCLLIIMECMEGGELFSRIQERGDQAFTEREAAEIMRDIGTAIQFLHSHNIAHRDVKPENLLYTSKEKDAVLKLTDFGFAKETTQNALQTPCYTPYYVAPEVLGPEKYDKSCDMWSLGVIMYILLCGFPPFYSNTGQAISPGMKRRIRLGQYGFPNPEWSEVSEDAKQLIRLLLKTDPTERLTITQFMNHPWINQSMVVPQTPLHTARVLQEDKDHWDEVKEEMTSALATMRVDYDQVKIKDLKTSNNRLLNKRRKKQAGSSSASQGCNNQ",
# MAPK5_HUMAN
"MSEESDMDKAIKETSILEEYSINWTQKLGAGISGPVRVCVKKSTQERFALKILLDRPKARNEVRLHMMCATHPNIVQIIEVFANSVQFPHESSPRARLLIVMEMMEGGELFHRISQHRHFTEKQASQVTKQIALALRHCHLLNIAHRDLKPENLLFKDNSLDAPVKLCDFGFAKIDQGDLMTPQFTPYYVAPQVLEAQRRHQKEKSGIIPTSPTPYTYNKSCDLWSLGVIIYVMLCGYPPFYSKHHSRTIPKDMRRKIMTGSFEFPEEEWSQISEMAKDVVRKLLKVKPEERLTIEGVLDHPWLNSTEALDNVLPSAQLMMDKAVVAGIQQAHAEQLANMRIQDLKVSLKPLHSVNNPILRKRKLLGTKPKDSVYIHDHENGAEDSNVALEKLRDVIAQCILPQAGKGENEDEKLNEVMQEAWKYNRECKLLRDTLQSFSWNGRGFTDKVDRLKLAEIVKQVIEEQTTSHESQ",
# MARK1_HUMAN
"MSARTPLPTVNERDTENHTSVDGYTEPHIQPTKSSSRQNIPRCRNSITSATDEQPHIGNYRLQKTIGKGNFAKVKLARHVLTGREVAVKIIDKTQLNPTSLQKLFREVRIMKILNHPNIVKLFEVIETEKTLYLVMEYASGGEVFDYLVAHGRMKEKEARAKFRQIVSAVQYCHQKYIVHRDLKAENLLLDGDMNIKIADFGFSNEFTVGNKLDTFCGSPPYAAPELFQGKKYDGPEVDVWSLGVILYTLVSGSLPFDGQNLKELRERVLRGKYRIPFYMSTDCENLLKKLLVLNPIKRGSLEQIMKDRWMNVGHEEEELKPYTEPDPDFNDTKRIDIMVTMGFARDEINDALINQKYDEVMATYILLGRKPPEFEGGESLSSGNLCQRSRPSSDLNNSTLQSPAHLKVQRSISANQKQRRFSDHAGPSIPPAVSYTKRPQANSVESEQKEEWDKDVARKLGSTTVGSKSEMTASPLVGPERKKSSTIPSNNVYSGGSMARRNTYVCERTTDRYVALQNGKDSSLTEMSVSSISSAGSSVASAVPSARPRHQKSMSTSGHPIKVTLPTIKDGSEAYRPGTTQRVPAASPSAHSISTATPDRTRFPRGSSSRSTFHGEQLRERRSVAYNGPPASPSHETGAFAHARRGTSTGIISKITSKFVRRDPSEGEASGRTDTSRSTSGEPKERDKEEGKDSKPRSLRFTWSMKTTSSMDPNDMMREIRKVLDANNCDYEQKERFLLFCVHGDARQDSLVQWEMEVCKLPRLSLNGVRFKRISGTSIAFKNIASKIANELKL",
# MAST2_HUMAN
"MKRSRCRDRPQPPPPDRREDGVQRAAELSQSLPPRRRAPPGRQRLEERTGPAGPEGKEQDVVTGVSPLLFRKLSNPDIFSSTGKVKLQRQLSQDDCKLWRGNLASSLSGKQLLPLSSSVHSSVGQVTWQSSGEASNLVRMRNQSLGQSAPSLTAGLKELSLPRRGSFCRTSNRKSLIVTSSTSPTLPRPHSPLHGHTGNSPLDSPRNFSPNAPAHFSFVPARRTDGRRWSLASLPSSGYGTNTPSSTVSSSCSSQEKLHQLPFQPTADELHFLTKHFSTESVPDEEGRQSPAMRPRSRSLSPGRSPVSFDSEIIMMNHVYKERFPKATAQMEERLAEFISSNTPDSVLPLADGALSFIHHQVIEMARDCLDKSRSGLITSQYFYELQDNLEKLLQDAHERSESSEVAFVMQLVKKLMIIIARPARLLECLEFDPEEFYHLLEAAEGHAKEGQGIKCDIPRYIVSQLGLTRDPLEEMAQLSSCDSPDTPETDDSIEGHGASLPSKKTPSEEDFETIKLISNGAYGAVFLVRHKSTRQRFAMKKINKQNLILRNQIQQAFVERDILTFAENPFVVSMFCSFDTKRHLCMVMEYVEGGDCATLLKNIGALPVDMVRLYFAETVLALEYLHNYGIVHRDLKPDNLLITSMGHIKLTDFGLSKIGLMSLTTNLYEGHIEKDAREFLDKQVCGTPEYIAPEVILRQGYGKPVDWWAMGIILYEFLVGCVPFFGDTPEELFGQVISDEIVWPEGDEALPPDAQDLTSKLLHQNPLERLGTGSAYEVKQHPFFTGLDWTGLLRQKAEFIPQLESEDDTSYFDTRSERYHHMDSEDEEEVSEDGCLEIRQFSSCSPRFNKVYSSMERLSLLEERRTPPPTKRSLSEEKEDHSDGLAGLKGRDRSWVIGSPEILRKRLSVSESSHTESDSSPPMTVRRRCSGLLDAPRFPEGPEEASSTLRRQPQEGIWVLTPPSGEGVSGPVTEHSGEQRPKLDEEAVGRSSGSSPAMETRGRGTSQLAEGATAKAISDLAVRRARHRLLSGDSTEKRTARPVNKVIKSASATALSLLIPSEHHTCSPLASPMSPHSQSSNPSSRDSSPSRDFLPALGSMRPPIIIHRAGKKYGFTLRAIRVYMGDSDVYTVHHMVWHVEDGGPASEAGLRQGDLITHVNGEPVHGLVHTEVVELILKSGNKVAISTTPLENTSIKVGPARKGSYKAKMARRSKRSRGKDGQESRKRSSLFRKITKQASLLHTSRSLSSLNRSLSSGESGPGSPTHSHSLSPRSPTQGYRVTPDAVHSVGGNSSQSSSPSSSVPSSPAGSGHTRPSSLHGLAPKLQRQYRSPRRKSAGSIPLSPLAHTPSPPPPTASPQRSPSPLSGHVAQAFPTKLHLSPPLGRQLSRPKSAEPPRSPLLKRVQSAEKLAAALAASEKKLATSRKHSLDLPHSELKKELPPREVSPLEVVGARSVLSGKGALPGKGVLQPAPSRALGTLRQDRAERRESLQKQEAIREVDSSEDDTEEGPENSQGAQELSLAPHPEVSQSVAPKGAGESGEEDPFPSRDPRSLGPMVPSLLTGITLGPPRMESPSGPHRRLGSPQAIEEAASSSSAGPNLGQSGATDPIPPEGCWKAQHLHTQALTALSPSTSGLTPTSSCSPPSSTSGKLSMWSWKSLIEGPDRASPSRKATMAGGLANLQDLENTTPAQPKNLSPREQGKTQPPSAPRLAHPSYEDPSQGWLWESECAQAVKEDPALSITQVPDASGDRRQDVPCRGCPLTQKSEPSLRRGQEPGGHQKHRDLALVPDELLKQT",
# MP2K1_HUMAN
"MPKKKPTPIQLNPAPDGSAVNGTSSAETNLEALQKKLEELELDEQQRKRLEAFLTQKQKVGELKDDDFEKISELGAGNGGVVFKVSHKPSGLVMARKLIHLEIKPAIRNQIIRELQVLHECNSPYIVGFYGAFYSDGEISICMEHMDGGSLDQVLKKAGRIPEQILGKVSIAVIKGLTYLREKHKIMHRDVKPSNILVNSRGEIKLCDFGVSGQLIDSMANSFVGTRSYMSPERLQGTHYSVQSDIWSMGLSLVEMAVGRYPIPPPDAKELELMFGCQVEGDAAETPPRPRTPGRPLSSYGMDSRPPMAIFELLDYIVNEPPPKLPSGVFSLEFQDFVNKCLIKNPAERADLKQLMVHAFIKRSDAEEVDFAGWLCSTIGLNQPSTPTHAAGV",
# MP2K2_HUMAN
"MLARRKPVLPALTINPTIAEGPSPTSEGASEANLVDLQKKLEELELDEQQKKRLEAFLTQKAKVGELKDDDFERISELGAGNGGVVTKVQHRPSGLIMARKLIHLEIKPAIRNQIIRELQVLHECNSPYIVGFYGAFYSDGEISICMEHMDGGSLDQVLKEAKRIPEEILGKVSIAVLRGLAYLREKHQIMHRDVKPSNILVNSRGEIKLCDFGVSGQLIDSMANSFVGTRSYMAPERLQGTHYSVQSDIWSMGLSLVELAVGRYPIPPPDAKELEAIFGRPVVDGEEGEPHSISPRPRPPGRPVSGHGMDSRPAMAIFELLDYIVNEPPPKLPNGVFTPDFQEFVNKCLIKNPAERADLKMLTNHTFIKRSEVEEVDFAGWLCKTLRLNQPGTPTRTAV",
# MP2K5_HUMAN
"MLWLALGPFPAMENQVLVIRIKIPNSGAVDWTVHSGPQLLFRDVLDVIGQVLPEATTTAFEYEDEDGDRITVRSDEEMKAMLSYYYSTVMEQQVNGQLIEPLQIFPRACKPPGERNIHGLKVNTRAGPSQHSSPAVSDSLPSNSLKKSSAELKKILANGQMNEQDIRYRDTLGHGNGGTVYKAYHVPSGKILAVKVILLDITLELQKQIMSELEILYKCDSSYIIGFYGAFFVENRISICTEFMDGGSLDVYRKMPEHVLGRIAVAVVKGLTYLWSLKILHRDVKPSNMLVNTRGQVKLCDFGVSTQLVNSIAKTYVGTNAYMAPERISGEQYGIHSDVWSLGISFMELALGRFPYPQIQKNQGSLMPLQLLQCIVDEDSPVLPVGEFSEPFVHFITQCMRKQPKERPAPEELMGHPFIVQFNDGNAAVVSMWVCRALEERRSQQGPP",
# MP2K6_HUMAN
"MSQSKGKKRNPGLKIPKEAFEQPQTSSTPPRDLDSKACISIGNQNFEVKADDLEPIMELGRGAYGVVEKMRHVPSGQIMAVKRIRATVNSQEQKRLLMDLDISMRTVDCPFTVTFYGALFREGDVWICMELMDTSLDKFYKQVIDKGQTIPEDILGKIAVSIVKALEHLHSKLSVIHRDVKPSNVLINALGQVKMCDFGISGYLVDSVAKTIDAGCKPYMAPERINPELNQKGYSVKSDIWSLGITMIELAILRFPYDSWGTPFQQLKQVVEEPSPQLPADKFSAEFVDFTSQCLKKNSKERPTYPELMQHPFFTLHESKGTDVASFVKLILGD",
# M3K1_HUMAN
"MAAAAGNRASSSGFPGARATSPEAGGGGGALKASSAPAAAAGLLREAGSGGRERADWRRRQLRKVRSVELDQLPEQPLFLAASPPASSTSPSPEPADAAGSGTGFQPVAVPPPHGAASRGGAHLTESVAAPDSGASSPAAAEPGEKRAPAAEPSPAAAPAGREMENKETLKGLHKMDDRPEERMIREKLKATCMPAWKHEWLERRNRRGPVVVKPIPVKGDGSEMNHLAAESPGEVQASAASPASKGRRSPSPGNSPSGRTVKSESPGVRRKRVSPVPFQSGRITPPRRAPSPDGFSPYSPEETNRRVNKVMRARLYLLQQIGPNSFLIGGDSPDNKYRVFIGPQNCSCARGTFCIHLLFVMLRVFQLEPSDPMLWRKTLKNFEVESLFQKYHSRRSSRIKAPSRNTIQKFVSRMSNSHTLSSSSTSTSSSENSIKDEEEQMCPICLLGMLDEESLTVCEDGCRNKLHHHCMSIWAEECRRNREPLICPLCRSKWRSHDFYSHELSSPVDSPSSLRAAQQQTVQQQPLAGSRRNQESNFNLTHYGTQQIPPAYKDLAEPWIQVFGMELVGCLFSRNWNVREMALRRLSHDVSGALLLANGESTGNSGGSSGSSPSGGATSGSSQTSISGDVVEACCSVLSMVCADPVYKVYVAALKTLRAMLVYTPCHSLAERIKLQRLLQPVVDTILVKCADANSRTSQLSISTLLELCKGQAGELAVGREILKAGSIGIGGVDYVLNCILGNQTESNNWQELLGRLCLIDRLLLEFPAEFYPHIVSTDVSQAEPVEIRYKKLLSLLTFALQSIDNSHSMVGKLSRRIYLSSARMVTTVPHVFSKLLEMLSVSSSTHFTRMRRRLMAIADEVEIAEAIQLGVEDTLDGQQDSFLQASVPNNYLETTENSSPECTVHLEKTGKGLCATKLSASSEDISERLASISVGPSSSTTTTTTTTEQPKPMVQTKGRPHSQCLNSSPLSHHSQLMFPALSTPSSSTPSVPAGTATDVSKHRLQGFIPCRIPSASPQTQRKFSLQFHRNCPENKDSDKLSPVFTQSRPLPSSNIHRPKPSRPTPGNTSKQGDPSKNSMTLDLNSSSKCDDSFGCSSNSSNAVIPSDETVFTPVEEKCRLDVNTELNSSIEDLLEASMPSSDTTVTFKSEVAVLSPEKAENDDTYKDDVNHNQKCKEKMEAEEEEALAIAMAMSASQDALPIVPQLQVENGEDIIIIQQDTPETLPGHTKAKQPYREDTEWLKGQQIGLGAFSSCYQAQDVGTGTLMAVKQVTYVRNTSSEQEEVVEALREEIRMMSHLNHPNIIRMLGATCEKSNYNLFIEWMAGGSVAHLLSKYGAFKESVVINYTEQLLRGLSYLHENQIIHRDVKGANLLIDSTGQRLRIADFGAAARLASKGTGAGEFQGQLLGTIAFMAPEVLRGQQYGRSCDVWSVGCAIIEMACAKPPWNAEKHSNHLALIFKIASATTAPSIPSHLSPGLRDVALRCLELQPQDRPPSRELLKHPVFRTTW",
# M3K2_HUMAN
"MDDQQALNSIMQDLAVLHKASRPALSLQETRKAKSSSPKKQNDVRVKFEHRGEKRILQFPRPVKLEDLRSKAKIAFGQSMDLHYTNNELVIPLTTQDDLDKAVELLDRSIHMKSLKILLVINGSTQATNLEPLPSLEDLDNTVFGAERKKRLSIIGPTSRDRSSPPPGYIPDELHQVARNGSFTSINSEGEFIPESMDQMLDPLSLSSPENSGSGSCPSLDSPLDGESYPKSRMPRAQSYPDNHQEFSDYDNPIFEKFGKGGTYPRRYHVSYHHQEYNDGRKTFPRARRTQGTSLRSPVSFSPTDHSLSTSSGSSIFTPEYDDSRIRRRGSDIDNPTLTVMDISPPSRSPRAPTNWRLGKLLGQGAFGRVYLCYDVDTGRELAVKQVQFDPDSPETSKEVNALECEIQLLKNLLHERIVQYYGCLRDPQEKTLSIFMEYMPGGSIKDQLKAYGALTENVTRKYTRQILEGVHYLHSNMIVHRDIKGANILRDSTGNVKLGDFGASKRLQTICLSGTGMKSVTGTPYWMSPEVISGEGYGRKADIWSVACTVVEMLTEKPPWAEFEAMAAIFKIATQPTNPKLPPHVSDYTRDFLKRIFVEAKLRPSADELLRHMFVHYH",
# M3K3_HUMAN
"MDEQEALNSIMNDLVALQMNRRHRMPGYETMKNKDTGHSNRQSDVRIKFEHNGERRIIAFSRPVKYEDVEHKVTTVFGQPLDLHYMNNELSILLKNQDDLDKAIDILDRSSSMKSLRILLLSQDRNHNSSSPHSGVSRQVRIKASQSAGDINTIYQPPEPRSRHLSVSSQNPGRSSPPPGYVPERQQHIARQGSYTSINSEGEFIPETSEQCMLDPLSSAENSLSGSCQSLDRSADSPSFRKSRMSRAQSFPDNRQEYSDRETQLYDKGVKGGTYPRRYHVSVHHKDYSDGRRTFPRIRRHQGNLFTLVPSSRSLSTNGENMGLAVQYLDPRGRLRSADSENALSVQERNVPTKSPSAPINWRRGKLLGQGAFGRVYLCYDVDTGRELASKQVQFDPDSPETSKEVSALECEIQLLKNLQHERIVQYYGCLRDRAEKTLTIFMEYMPGGSVKDQLKAYGALTESVTRKYTRQILEGMSYLHSNMIVHRDIKGANILRDSAGNVKLGDFGASKRLQTICMSGTGMRSVTGTPYWMSPEVISGEGYGRKADVWSLGCTVVEMLTEKPPWAEYEAMAAIFKIATQPTNPQLPSHISEHGRDFLRRIFVEARQRPSAEELLTHHFAQLMY",
# M3K4_HUMAN
"MREAAAALVPPPAFAVTPAAAMEEPPPPPPPPPPPPEPETESEPECCLAARQEGTLGDSACKSPESDLEDFSDETNTENLYGTSPPSTPRQMKRMSTKHQRNNVGRPASRSNLKEKMNAPNQPPHKDTGKTVENVEEYSYKQEKKIRAALRTTERDRKKNVQCSFMLDSVGGSLPKKSIPDVDLNKPYLSLGCSNAKLPVSVPMPIARPARQTSRTDCPADRLKFFETLRLLLKLTSVSKKKDREQRGQENTSGFWLNRSNELIWLELQAWHAGRTINDQDFFLYTARQAIPDIINEILTFKVDYGSFAFVRDRAGFNGTSVEGQCKATPGTKIVGYSTHHEHLQRQRVSFEQVKRIMELLEYIEALYPSLQALQKDYEKYAAKDFQDRVQALCLWLNITKDLNQKLRIMGTVLGIKNLSDIGWPVFEIPSPRPSKGNEPEYEGDDTEGELKELESSTDESEEEQISDPRVPEIRQPIDNSFDIQSRDCISKKLERLESEDDSLGWGAPDWSTEAGFSRHCLTSIYRPFVDKALKQMGLRKLILRLHKLMDGSLQRARIALVKNDRPVEFSEFPDPMWGSDYVQLSRTPPSSEEKCSAVSWEELKAMDLPSFEPAFLVLCRVLLNVIHECLKLRLEQRPAGEPSLLSIKQLVRECKEVLKGGLLMKQYYQFMLQEVLEDLEKPDCNIDAFEEDLHKMLMVYFDYMRSWIQMLQQLPQASHSLKNLLEEEWNFTKEITHYIRGGEAQAGKLFCDIAGMLLKSTGSFLEFGLQESCAEFWTSADDSSASDEIRRSVIEISRALKELFHEARERASKALGFAKMLRKDLEIAAEFRLSAPVRDLLDVLKSKQYVKVQIPGLENLQMFVPDTLAEEKSIILQLLNAAAGKDCSKDSDDVLIDAYLLLTKHGDRARDSEDSWGTWEAQPVKVVPQVETVDTLRSMQVDNLLLVVMQSAHLTIQRKAFQQSIEGLMTLCQEQTSSQPVIAKALQQLKNDALELCNRISNAIDRVDHMFTSEFDAEVDESESVTLQQYYREAMIQGYNFGFEYHKEVVRLMSGEFRQKIGDKYISFARKWMNYVLTKCESGRGTRPRWATQGFDFLQAIEPAFISALPEDDFLSLQALMNECIGHVIGKPHSPVTGLYLAIHRNSPRPMKVPRCHSDPPNPHLIIPTPEGFSTRSMPSDARSHGSPAAAAAAAAAAVAASRPSPSGGDSVLPKSISSAHDTRGSSVPENDRLASIAAELQFRSLSRHSSPTEERDEPAYPRGDSSGSTRRSWELRTLISQSKDTASKLGPIEAIQKSVRLFEEKRYREMRRKNIIGQVCDTPKSYDNVMHVGLRKVTFKWQRGNKIGEGQYGKVYTCISVDTGELMAMKEIRFQPNDHKTIKETADELKIFEGIKHPNLVRYFGVELHREEMYIFMEYCDEGTLEEVSRLGLQEHVIRLYSKQITIAINVLHEHGIVHRDIKGANIFLTSSGLIKLGDFGCSVKLKNNAQTMPGEVNSTLGTAAYMAPEVITRAKGEGHGRAADIWSLGCVVIEMVTGKRPWHEYEHNFQIMYKVGMGHKPPIPERLSPEGKDFLSHCLESDPKMRWTASQLLDHSFVKVCTDEE",
# M3K5_HUMAN
"MSTEADEGITFSVPPFAPSGFCTIPEGGICRRGGAAAVGEGEEHQLPPPPPGSFWNVESAAAPGIGCPAATSSSSATRGRGSSVGGGSRRTTVAYVINEASQGQLVVAESEALQSLREACETVGATLETLHFGKLDFGETTVLDRFYNADIAVVEMSDAFRQPSLFYHLGVRESFSMANNIILYCDTNSDSLQSLKEIICQKNTMCTGNYTFVPYMITPHNKVYCCDSSFMKGLTELMQPNFELLLGPICLPLVDRFIQLLKVAQASSSQYFRESILNDIRKARNLYTGKELAAELARIRQRVDNIEVLTADIVINLLLSYRDIQDYDSIVKLVETLEKLPTFDLASHHHVKFHYAFALNRRNLPGDRAKALDIMIPMVQSEGQVASDMYCLVGRIYKDMFLDSNFTDTESRDHGASWFKKAFESEPTLQSGINYAVLLLAAGHQFESSFELRKVGVKLSSLLGKKGNLEKLQSYWEVGFFLGASVLANDHMRVIQASEKLFKLKTPAWYLKSIVETILIYKHFVKLTTEQPVAKQELVDFWMDFLVEATKTDVTVVRFPVLILEPTKIYQPSYLSINNEVEEKTISIWHVLPDDKKGIHEWNFSASSVRGVSISKFEERCCFLYVLHNSDDFQIYFCTELHCKKFFEMVNTITEEKGRSTEEGDCESDLLEYDYEYDENGDRVVLGKGTYGIVYAGRDLSNQVRIAIKEIPERDSRYSQPLHEEIALHKHLKHKNIVQYLGSFSENGFIKIFMEQVPGGSLSALLRSKWGPLKDNEQTIGFYTKQILEGLKYLHDNQIVHRDIKGDNVLINTYSGVLKISDFGTSKRLAGINPCTETFTGTLQYMAPEIIDKGPRGYGKAADIWSLGCTIIEMATGKPPFYELGEPQAAMFKVGMFKVHPEIPESMSAEAKAFILKCFEPDPDKRACANDLLVDEFLKVSSKKKKTQPKLSALSAGSNEYLRSISLPVPVLVEDTSSSSEYGSVSPDTELKVDPFSFKTRAKSCGERDVKGIRTLFLGIPDENFEDHSAPPSPEEKDSGFFMLRKDSERRATLHRILTEDQDKIVRNLMESLAQGAEEPKLKWEHITTLIASLREFVRSTDRKIIATTLSKLKLELDFDSHGISQVQVVLFGFQDAVNKVLRNHNIKPHWMFALDSIIRKAVQTAITILVPELRPHFSLASESDTADQEDLDVEDDHEEQPSNQTVRRPQAVIEDAVATSGVSTLSSTVSHDSQSAHRSLNVQLGRMKIETNRLLEELVRKEKELQALLHRAIEEKDQEIKHLKLKSQPIEIPELPVFHLNSSGTNTEDSELTDWLRVNGADEDTISRFLAEDYTLLDVLYYVTRDDLKCLRLRGGMLCTLWKAIIDFRNKQT",
# MERTK_HUMAN
"MGPAPLPLLLGLFLPALWRRAITEAREEAKPYPLFPGPFPGSLQTDHTPLLSLPHASGYQPALMFSPTQPGRPHTGNVAIPQVTSVESKPLPPLAFKHTVGHIILSEHKGVKFNCSISVPNIYQDTTISWWKDGKELLGAHHAITQFYPDDEVTAIIASFSITSVQRSDNGSYICKMKINNEEIVSDPIYIEVQGLPHFTKQPESMNVTRNTAFNLTCQAVGPPEPVNIFWVQNSSRVNEQPEKSPSVLTVPGLTEMAVFSCEAHNDKGLTVSKGVQINIKAIPSPPTEVSIRNSTAHSILISWVPGFDGYSPFRNCSIQVKEADPLSNGSVMIFNTSALPHLYQIKQLQALANYSIGVSCMNEIGWSAVSPWILASTTEGAPSVAPLNVTVFLNESSDNVDIRWMKPPTKQQDGELVGYRISHVWQSAGISKELLEEVGQNGSRARISVQVHNATCTVRIAAVTRGGVGPFSDPVKIFIPAHGWVDYAPSSTPAPGNADPVLIIFGCFCGFILIGLILYISLAIRKRVQETKFGNAFTEEDSELVVNYIAKKSFCRRAIELTLHSLGVSEELQNKLEDVVIDRNLLILGKILGEGEFGSVMEGNLKQEDGTSLKVAVKTMKLDNSSQREIEEFLSEAACMKDFSHPNVIRLLGVCIEMSSQGIPKPMVILPFMKYGDLHTYLLYSRLETGPKHIPLQTLLKFMVDIALGMEYLSNRNFLHRDLAARNCMLRDDMTVCVADFGLSKKIYSGDYYRQGRIAKMPVKWIAIESLADRVYTSKSDVWAFGVTMWEIATRGMTPYPGVQNHEMYDYLLHGHRLKQPEDCLDELYEIMYSCWRTDPLDRPTFSVLRLQLEKLLESLPDVRNQADVIYVNTQLLESSEGLAQGSTLAPLDLNIDPDSIIASCTPRAAISVVTAEVHDSKPHEGRYILNGGSEEWEDLTSAPSAAVTAEKNSVLPGERLVRNGVSWSHSSMLPLGSSLPDELLFADDSSEGSEVLM",
# MET_HUMAN
"MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNFTAETPIQNVILHEHHIFLGATNYIYVLNEEDLQKVAEYKTGPVLEHPDCFPCQDCSSKANLSGGVWKDNINMALVVDTYYDDQLISCGSVNRGTCQRHVFPHNHTADIQSEVHCIFSPQIEEPSQCPDCVVSALGAKVLSSVKDRFINFFVGNTINSSYFPDHPLHSISVRRLKETKDGFMFLTDQSYIDVLPEFRDSYPIKYVHAFESNNFIYFLTVQRETLDAQTFHTRIIRFCSINSGLHSYMEMPLECILTEKRKKRSTKKEVFNILQAAYVSKPGAQLARQIGASLNDDILFGVFAQSKPDSAEPMDRSAMCAFPIKYVNDFFNKIVNKNNVRCLQHFYGPNHEHCFNRTLLRNSSGCEARRDEYRTEFTTALQRVDLFMGQFSEVLLTSISTFIKGDLTIANLGTSEGRFMQVVVSRSGPSTPHVNFLLDSHPVSPEVIVEHTLNQNGYTLVITGKKITKIPLNGLGCRHFQSCSQCLSAPPFVQCGWCHDKCVRSEECLSGTWTQQICLPAIYKVFPNSAPLEGGTRLTICGWDFGFRRNNKFDLKKTRVLLGNESCTLTLSESTMNTLKCTVGPAMNKHFNMSIIISNGHGTTQYSTFSYVDPVITSISPKYGPMAGGTLLTLTGNYLNSGNSRHISIGGKTCTLKSVSNSILECYTPAQTISTEFAVKLKIDLANRETSIFSYREDPIVYEIHPTKSFISGGSTITGVGKNLNSVSVPRMVINVHEAGRNFTVACQHRSNSEIICCTTPSLQQLNLQLPLKTKAFFMLDGILSKYFDLIYVHNPVFKPFEKPVMISMGNENVLEIKGNDIDPEAVKGEVLKVGNKSCENIHLHSEAVLCTVPNDLLKLNSELNIEWKQAISSTVLGKVIVQPDQNFTGLIAGVVSISTALLLLLGFFLWLKKRKQIKDLGSELVRYDARVHTPHLDRLVSARSVSPTTEMVSNESVDYRATFPEDQFPNSSQNGSCRQVQYPLTDMSPILTSGDSDISSPLLQNTVHIDLSALNPELVQAVQHVVIGPSSLIVHFNEVIGRGHFGCVYHGTLLDNDGKKIHCAVKSLNRITDIGEVSQFLTEGIIMKDFSHPNVLSLLGICLRSEGSPLVVLPYMKHGDLRNFIRNETHNPTVKDLIGFGLQVAKGMKYLASKKFVHRDLAARNCMLDEKFTVKVADFGLARDMYDKEYYSVHNKTGAKLPVKWMALESLQTQKFTTKSDVWSFGVLLWELMTRGAPPYPDVNTFDITVYLLQGRRLLQPEYCPDPLYEVMLKCWHPKAEMRPSFSELVSRISAIFSTFIGEHYVHVNATYVNVKCVAPYPSLLSSEDNADDEVDTRPASFWETS",
# AMHR2_HUMAN
"MLGSLGLWALLPTAVEAPPNRRTCVFFEAPGVRGSTKTLGELLDTGTELPRAIRCLYSRCCFGIWNLTQDRAQVEMQGCRDSDEPGCESLHCDPSPRAHPSPGSTLFTCSCGTDFCNANYSHLPPPGSPGTPGSQGPQAAPGESIWMALVLLGLFLLLLLLLGSIILALLQRKNYRVRGEPVPEPRPDSGRDWSVELQELPELCFSQVIREGGHAVVWAGQLQGKLVAIKAFPPRSVAQFQAERALYELPGLQHDHIVRFITASRGGPGRLLSGPLLVLELHPKGSLCHYLTQYTSDWGSSLRMALSLAQGLAFLHEERWQNGQYKPGIAHRDLSSQNVLIREDGSCAIGDLGLALVLPGLTQPPAWTPTQPQGPAAIMEAGTQRYMAPELLDKTLDLQDWGMALRRADIYSLALLLWEILSRCPDLRPDSSPPPFQLAYEAELGNTPTSDELWALAVQERRRPYIPSTWRCFATDPDGLRELLEDCWDADPEARLTAECVQQRLAALAHPQESHPFPESCPRGCPPLCPEDCTSIPAPTILPCRPQRSACHFSVQQGPCSRNPQPACTLSPV",
# MP2K7_HUMAN
"MAASSLEQKLSRLEAKLKQENREARRRIDLNLDISPQRPRPTLQLPLANDGGSRSPSSESSPQHPTPPARPRHMLGLPSTLFTPRSMESIEIDQKLQEIMKQTGYLTIGGQRYQAEINDLENLGEMGSGTCGQVWKMRFRKTGHVIAVKQMRRSGNKEENKRILMDLDVVLKSHDCPYIVQCFGTFITNTDVFIAMELMGTCAEKLKKRMQGPIPERILGKMTVAIVKALYYLKEKHGVIHRDVKPSNILLDERGQIKLCDFGISGRLVDSKAKTRSAGCAAYMAPERIDPPDPTKPDYDIRADVWSLGISLVELATGQFPYKNCKTDFEVLTKVLQEEPPLLPGHMGFSGDFQSFVKDCLTKDHRKRPKYNKLLEHSFIKRYETLEVDVASWFKDVMAKTESPRTSGVLSQPHLPFFR",
# MYLK_HUMAN
"MGDVKLVASSHISKTSLSVDPSRVDSMPLTEAPAFILPPRNLCIKEGATAKFEGRVRGYPEPQVTWHRNGQPITSGGRFLLDCGIRGTFSLVIHAVHEEDRGKYTCEATNGSGARQVTVELTVEGSFAKQLGQPVVSKTLGDRFSAPAVETRPSIWGECPPKFATKLGRVVVKEGQMGRFSCKITGRPQPQVTWLKGNVPLQPSARVSVSEKNGMQVLEIHGVNQDDVGVYTCLVVNGSGKASMSAELSIQGLDSANRSFVRETKATNSDVRKEVTNVISKESKLDSLEAAAKSKNCSSPQRGGSPPWAANSQPQPPRESKLESCKDSPRTAPQTPVLQKTSSSITLQAARVQPEPRAPGLGVLSPSGEERKRPAPPRPATFPTRQPGLGSQDVVSKAANRRIPMEGQRDSAFPKFESKPQSQEVKENQTVKFRCEVSGIPKPEVAWFLEGTPVRRQEGSIEVYEDAGSHYLCLLKARTRDSGTYSCTASNAQGQLSCSWTLQVERLAVMEVAPSFSSVLKDCAVIEGQDFVLQCSVRGTPVPRITWLLNGQPIQYARSTCEAGVAELHIQDALPEDHGTYTCLAENALGQVSCSAWVTVHEKKSSRKSEYLLPVAPSKPTAPIFLQGLSDLKVMDGSQVTMTVQVSGNPPPEVIWLHNGNEIQESEDFHFEQRGTQHSLCIQEVFPEDTGTYTCEAWNSAGEVRTQAVLTVQEPHDGTQPWFISKPRSVTASLGQSVLISCAIAGDPFPTVHWLRDGKALCKDTGHFEVLQNEDVFTLVLKKVQPWHAGQYEILLKNRVGECSCQVSLMLQNSSARALPRGREPASCEDLCGGGVGADGGGSDRYGSLRPGWPARGQGWLEEEDGEDVRGVLKRRVETRQHTEEAIRQQEVEQLDFRDLLGKKVSTKTLSEDDLKEIPAEQMDFRANLQRQVKPKTVSEEERKVHSPQQVDFRSVLAKKGTSKTPVPEKVPPPKPATPDFRSVLGGKKKLPAENGSSSAETLNAKAVESSKPLSNAQPSGPLKPVGNAKPAETLKPMGNAKPAETLKPMGNAKPDENLKSASKEELKKDVKNDVNCKRGHAGTTDNEKRSESQGTAPAFKQKLQDVHVAEGKKLLLQCQVSSDPPATIIWTLNGKTLKTTKFIILSQEGSLCSVSIEKALPEDRGLYKCVAKNDAGQAECSCQVTVDDAPASENTKAPEMKSRRPKSSLPPVLGTESDATVKKKPAPKTPPKAAMPPQIIQFPEDQKVRAGESVELFGKVTGTQPITCTWMKFRKQIQESEHMKVENSENGSKLTILAARQEHCGCYTLLVENKLGSRQAQVNLTVVDKPDPPAGTPCASDIRSSSLTLSWYGSSYDGGSAVQSYSIEIWDSANKTWKELATCRSTSFNVQDLLPDHEYKFRVRAINVYGTSEPSQESELTTVGEKPEEPKDEVEVSDDDEKEPEVDYRTVTINTEQKVSDFYDIEERLGSGKFGQVFRLVEKKTRKVWAGKFFKAYSAKEKENIRQEISIMNCLHHPKLVQCVDAFEEKANIVMVLEIVSGGELFERIIDEDFELTERECIKYMRQISEGVEYIHKQGIVHLDLKPENIMCVNKTGTRIKLIDFGLARRLENAGSLKVLFGTPEFVAPEVINYEPIGYATDMWSIGVICYILVSGLSPFMGDNDNETLANVTSATWDFDDEAFDEISDDAKDFISNLLKKDMKNRLDCTQCLQHPWLMKDTKNMEAKKLSKDRMKKYMARRKWQKTGNAVRAIGRLSSMAMISGLSGRKSSTGSPTSPLNAEKLESEEDVSQAFLEAVAEEKPHVKPYFSKTIRDLEVVEGSAARFDCKIEGYPDPEVVWFKDDQSIRESRHFQIDYDEDGNCSLIISDVCGDDDAKYTCKAVNSLGEATCTAELIVETMEEGEGEGEEEEE",
# M3K9_HUMAN
"MEPSRALLGCLASAAAAAPPGEDGAGAGAEEEEEEEEEAAAAVGPGELGCDAPLPYWTAVFEYEAAGEDELTLRLGDVVEVLSKDSQVSGDEGWWTGQLNQRVGIFPSNYVTPRSAFSSRCQPGGEDPSCYPPIQLLEIDFAELTLEEIIGIGGFGKVYRAFWIGDEVAVKAARHDPDEDISQTIENVRQEAKLFAMLKHPNIIALRGVCLKEPNLCLVMEFARGGPLNRVLSGKRIPPDILVNWAVQIARGMNYLHDEAIVPIIHRDLKSSNILILQKVENGDLSNKILKITDFGLAREWHRTTKMSAAGTYAWMAPEVIRASMFSKGSDVWSYGVLLWELLTGEVPFRGIDGLAVAYGVAMNKLALPIPSTCPEPFAKLMEDCWNPDPHSRPSFTNILDQLTTIEESGFFEMPKDSFHCLQDNWKHEIQEMFDQLRAKEKELRTWEEELTRAALQQKNQEELLRRREQELAEREIDILERELNIIIHQLCQEKPRVKKRKGKFRKSRLKLKDGNRISLPSDFQHKFTVQASPTMDKRKSLINSRSSPPASPTIIPRLRAIQLTPGESSKTWGRSSVVPKEEGEEEEKRAPKKKGRTWGPGTLGQKELASGDEGSPQRREKANGLSTPSESPHFHLGLKSLVDGYKQWSSSAPNLVKGPRSSPALPGFTSLMEMEDEDSEGPGSGESRLQHSPSQSYLCIPFPRGEDGDGPSSDGIHEEPTPVNSATSTPQLTPTNSLKRGGAHHRRCEVALLGCGAVLAATGLGFDLLEAGKCQLLPLEEPEPPAREEKKRREGLFQRSSRPRRSTSPPSRKLFKKEEPMLLLGDPSASLTLLSLSSISECNSTRSLLRSDSDEIVVYEMPVSPVEAPPLSPCTHNPLVNVRVERFKRDPNQSLTPTHVTLTTPSQPSSHRRTPSDGALKPETLLASRSPSSNGLSPSPGAGMLKTPSPSRDPGEFPRLPDPNVVFPPTPRRWNTQQDSTLERPKTLEFLPRPRPSANRQRLDPWWFVSPSHARSTSPANSSSTETPSNLDSCFASSSSTVEERPGLPALLPFQAGPLPPTERTLLDLDAEGQSQDSTVPLCRAELNTHRPAPYEIQQEFWS",
# M3K10_HUMAN
"MEEEEGAVAKEWGTTPAGPVWTAVFDYEAAGDEELTLRRGDRVQVLSQDCAVSGDEGWWTGQLPSGRVGVFPSNYVAPGAPAAPAGLQLPQEIPFHELQLEEIIGVGGFGKVYRALWRGEEVAVKAARLDPEKDPAVTAEQVCQEARLFGALQHPNIIALRGACLNPPHLCLVMEYARGGALSRVLAGRRVPPHVLVNWAVQVARGMNYLHNDAPVPIIHRDLKSINILILEAIENHNLADTVLKITDFGLAREWHKTTKMSAAGTYAWMAPEVIRLSLFSKSSDVWSFGVLLWELLTGEVPYREIDALAVAYGVAMNKLTLPIPSTCPEPFARLLEECWDPDPHGRPDFGSILKRLEVIEQSALFQMPLESFHSLQEDWKLEIQHMFDDLRTKEKELRSREEELLRAAQEQRFQEEQLRRREQELAEREMDIVERELHLLMCQLSQEKPRVRKRKGNFKRSRLLKLREGGSHISLPSGFEHKITVQASPTLDKRKGSDGASPPASPSIIPRLRAIRLTPVDCGGSSSGSSSGGSGTWSRGGPPKKEELVGGKKKGRTWGPSSTLQKERVGGEERLKGLGEGSKQWSSSAPNLGKSPKHTPIAPGFASLNEMEEFAEAEDGGSSVPPSPYSTPSYLSVPLPAEPSPGARAPWEPTPSAPPARWGHGARRRCDLALLGCATLLGAVGLGADVAEARAADGEEQRRWLDGLFFPRAGRFPRGLSPPARPHGRREDVGPGLGLAPSATLVSLSSVSDCNSTRSLLRSDSDEAAPAAPSPPPSPPAPTPTPSPSTNPLVDLELESFKKDPRQSLTPTHVTAACAVSRGHRRTPSDGALGQRGPPEPAGHGPGPRDLLDFPRLPDPQALFPARRRPPEFPGRPTTLTFAPRPRPAASRPRLDPWKLVSFGRTLTISPPSRPDTPESPGPPSVQPTLLDMDMEGQNQDSTVPLCGAHGSH",
# DYR1A_HUMAN
"MHTGGETSACKPSSVRLAPSFSFHAAGLQMAGQMPHSHQYSDRRQPNISDQQVSALSYSDQIQQPLTNQVMPDIVMLQRRMPQTFRDPATAPLRKLSVDLIKTYKHINEVYYAKKKRRHQQGQGDDSSHKKERKVYNDGYDDDNYDYIVKNGEKWMDRYEIDSLIGKGSFGQVVKAYDRVEQEWVAIKIIKNKKAFLNQAQIEVRLLELMNKHDTEMKYYIVHLKRHFMFRNHLCLVFEMLSYNLYDLLRNTNFRGVSLNLTRKFAQQMCTALLFLATPELSIIHCDLKPENILLCNPKRSAIKIVDFGSSCQLGQRIYQYIQSRFYRSPEVLLGMPYDLAIDMWSLGCILVEMHTGEPLFSGANEVDQMNKIVEVLGIPPAHILDQAPKARKFFEKLPDGTWNLKKTKDGKREYKPPGTRKLHNILGVETGGPGGRRAGESGHTVADYLKFKDLILRMLDYDPKTRIQPYYALQHSFFKKTADEGTNTSNSVSTSPAMEQSQSSGTTSSTSSSSGGSSGTSNSGRARSDPTHQHRHSGGHFTAAVQAMDCETHSPQVRQQFPAPLGWSGTEAPTQVTVETHPVQETTFHVAPQQNALHHHHGNSSHHHHHHHHHHHHHGQQALGNRTRPRVYNSPTNSSSTQDSMEVGHSHHSMTSLSSSTTSSSTSSSSTGNQGNQAYQNRPVAANTLDFGQNGAMDVNLTVYSNPRQETGIAGHPTYQFSANTGPAHYMTEGHLTMRQGADREESPMTGVCVQQSPVASS",
# MKNK1_HUMAN
"MVSSQKLEKPIEMGSSEPLPIADGDRRRKKKRRGRATDSLPGKFEDMYKLTSELLGEGAYAKVQGAVSLQNGKEYAVKIIEKQAGHSRSRVFREVETLYQCQGNKNILELIEFFEDDTRFYLVFEKLQGGSILAHIQKQKHFNEREASRVVRDVAAALDFLHTKDKVSLCHLGWSAMAPSGLTAAPTSLGSSDPPTSASQVAGTTGIAHRDLKPENILCESPEKVSPVKICDFDLGSGMKLNNSCTPITTPELTTPCGSAEYMAPEVVEVFTDQATFYDKRCDLWSLGVVLYIMLSGYPPFVGHCGADCGWDRGEVCRVCQNKLFESIQEGKYEFPDKDWAHISSEAKDLISKLLVRDAKQRLSAAQVLQHPWVQGQAPEKGLPTPQVLQRNSSTMDLTLFAAEAIALNRQLSQHEENELAEEPEALADGLCSMKLSPPCKSRLARRRALAQAGRGEDRSPPTAL",
# MKNK2_HUMAN
"MVQKKPAELQGFHRSFKGQNPFELAFSLDQPDHGDSDFGLQCSARPDMPASQPIDIPDAKKRGKKKKRGRATDSFSGRFEDVYQLQEDVLGEGAHARVQTCINLITSQEYAVKIIEKQPGHIRSRVFREVEMLYQCQGHRNVLELIEFFEEEDRFYLVFEKMRGGSILSHIHKRRHFNELEASVVVQDVASALDFLHNKGIAHRDLKPENILCEHPNQVSPVKICDFDLGSGIKLNGDCSPISTPELLTPCGSAEYMAPEVVEAFSEEASIYDKRCDLWSLGVILYILLSGYPPFVGRCGSDCGWDRGEACPACQNMLFESIQEGKYEFPDKDWAHISCAAKDLISKLLVRDAKQRLSAAQVLQHPWVQGCAPENTLPTPMVLQRNSCAKDLTSFAAEAIAMNRQLAQHDEDLAEEEAAGQGQPVLVRATSRCLQLSPPSQSKLAQRRQRASLSSAPVVLVGDHA",
# MOS_HUMAN
"MPSPLALRPYLRSEFSPSVDARPCSSPSELPAKLLLGATLPRAPRLPRRLAWCSIDWEQVCLLQRLGAGGFGSVYKATYRGVPVAIKQVNKCTKNRLASRRSFWAELNVARLRHDNIVRVVAASTRTPAGSNSLGTIIMEFGGNVTLHQVIYGAAGHPEGDAGEPHCRTGGQLSLGKCLKYSLDVVNGLLFLHSQSIVHLDLKPANILISEQDVCKISDFGCSEKLEDLLCFQTPSYPLGGTYTHRAPELLKGEGVTPKADIYSFAITLWQMTTKQAPYSGERQHILYAVVAYDLRPSLSAAVFEDSLPGQRLGDVIQRCWRPSAAQRPSARLLLVDLTSLKAELG",
# MP2K3_HUMAN
"MESPASSQPASMPQSKGKSKRKKDLRISCMSKPPAPNPTPPRNLDSRTFITIGDRNFEVEADDLVTISELGRGAYGVVEKVRHAQSGTIMAVKRIRATVNSQEQKRLLMDLDINMRTVDCFYTVTFYGALFREGDVWICMELMDTSLDKFYRKVLDKNMTIPEDILGEIAVSIVRALEHLHSKLSVIHRDVKPSNVLINKEGHVKMCDFGISGYLVDSVAKTMDAGCKPYMAPERINPELNQKGYNVKSDVWSLGITMIEMAILRFPYESWGTPFQQLKQVVEEPSPQLPADRFSPEFVDFTAQCLRKNPAERMSYLELMEHPFFTLHKTKKTDIAAFVKEILGEDS",
# MP2K4_HUMAN
"MAAPSPSGGGGSGGGSGSGTPGPVGSPAPGHPAVSSMQGKRKALKLNFANPPFKSTARFTLNPNPTGVQNPHIERLRTHSIESSGKLKISPEQHWDFTAEDLKDLGEIGRGAYGSVNKMVHKPSGQIMAVKRIRSTVDEKEQKQLLMDLDVVMRSSDCPYIVQFYGALFREGDCWICMELMSTSFDKFYKYVYSVLDDVIPEEILGKITLATVKALNHLKENLKIIHRDIKPSNILLDRSGNIKLCDFGISGQLVDSIAKTRDAGCRPYMAPERIDPSASRQGYDVRSDVWSLGITLYELATGRFPYPKWNSVFDQLTQVVKGDPPQLSNSEEREFSPSFINFVNLCLTKDESKRPKYKELLKHPFILMYEERAVEVACYVCKILDQMPATPSSPMYVD",
# MRCKB_HUMAN
"MSAKVRLKKLEQLLLDGPWRNESALSVETLLDVLVCLYTECSHSALRRDKYVAEFLEWAKPFTQLVKEMQLHREDFEIIKVIGRGAFGEVAVVKMKNTERIYAMKILNKWEMLKRAETACFREERDVLVNGDCQWITALHYAFQDENHLYLVMDYYVGGDLLTLLSKFEDKLPEDMARFYIGEMVLAIDSIHQLHYVHRDIKPDNVLLDVNGHIRLADFGSCLKMNDDGTVQSSVAVGTPDYISPEILQAMEDGMGKYGPECDWWSLGVCMYEMLYGETPFYAESLVETYGKIMNHEERFQFPSHVTDVSEEAKDLIQRLICSRERRLGQNGIEDFKKHAFFEGLNWENIRNLEAPYIPDVSSPSDTSNFDVDDDVLRNTEILPPGSHTGFSGLHLPFIGFTFTTESCFSDRGSLKSIMQSNTLTKDEDVQRDLEHSLQMEAYERRIRRLEQEKLELSRKLQESTQTVQSLHGSSRALSNSNRDKEIKKLNEEIERLKNKIADSNRLERQLEDTVALRQEREDSTQRLRGLEKQHRVVRQEKEELHKQLVEASERLKSQAKELKDAHQQRKLALQEFSELNERMAELRAQKQKVSRQLRDKEEEMEVATQKVDAMRQEMRRAEKLRKELEAQLDDAVAEASKERKLREHSENFCKQMESELEALKVKQGGRGAGATLEHQQEISKIKSELEKKVLFYEEELVRREASHVLEVKNVKKEVHDSESHQLALQKEILMLKDKLEKSKRERHNEMEEAVGTIKDKYERERAMLFDENKKLTAENEKLCSFVDKLTAQNRQLEDELQDLAAKKESVAHWEAQIAEIIQWVSDEKDARGYLQALASKMTEELEALRSSSLGSRTLDPLWKVRRSQKLDMSARLELQSALEAEIRAKQLVQEELRKVKDANLTLESKLKDSEAKNRELLEEMEILKKKMEEKFRADTGLKLPDFQDSIFEYFNTAPLAHDLTFRTSSASEQETQAPKPEASPSMSVAASEQQEDMARPPQRPSAVPLPTTQALALAGPKPKAHQFSIKSFSSPTQCSHCTSLMVGLIRQGYACEVCSFACHVSCKDGAPQVCPIPPEQSKRPLGVDVQRGIGTAYKGHVKVPKPTGVKKGWQRAYAVVCDCKLFLYDLPEGKSTQPGVIASQVLDLRDDEFSVSSVLASDVIHATRRDIPCIFRVTASLLGAPSKTSSLLILTENENEKRKWVGILEGLQSILHKNRLRNQVVHVPLEAYDSSLPLIKAILTAAIVDADRIAVGLEEGLYVIEVTRDVIVRAADCKKVHQIELAPREKIVILLCGRNHHVHLYPWSSLDGAEGSFDIKLPETKGCQLMATATLKRNSGTCLFVAVKRLILCYEIQRTKPFHRKFNEIVAPGSVQCLAVLRDRLCVGYPSGFCLLSIQGDGQPLNLVNPNDPSLAFLSQQSFDALCAVELESEEYLLCFSHMGLYVDPQGRRARAQELMWPAAPVACSCSPTHVTVYSEYGVDVFDVRTMEWVQTIGLRRIRPLNSEGTLNLLNCEPPRLIYFKSKFSGAVLNVPDTSDNSKKQMLRTRSKRRFVFKVPEEERLQQRREMLRDPELRSKMISNPTNFNHVAHMGPGDGMQVLMDLPLSAVPPSQEERPGPAPTNLARQPPSRNKPYISWPSSGGSEPSVTVPLRSMSDPDQDFDKEPDSDSTKHSTPSNSSNPSGPPSPNSPHRSQLPLEGLEQPACDT",
# KS6A5_HUMAN
"MEEEGGSSGGAAGTSADGGDGGEQLLTVKHELRTANLTGHAEKVGIENFELLKVLGTGAYGKVFLVRKISGHDTGKLYAMKVLKKATIVQKAKTTEHTRTERQVLEHIRQSPFLVTLHYAFQTETKLHLILDYINGGELFTHLSQRERFTEHEVQIYVGEIVLALEHLHKLGIIYRDIKLENILLDSNGHVVLTDFGLSKEFVADETERAYSFCGTIEYMAPDIVRGGDSGHDKAVDWWSLGVLMYELLTGASPFTVDGEKNSQAEISRRILKSEPPYPQEMSALAKDLIQRLLMKDPKKRLGCGPRDADEIKEHLFFQKINWDDLAAKKVPAPFKPVIRDELDVSNFAEEFTEMDPTYSPAALPQSSEKLFQGYSFVAPSILFKRNAAVIDPLQFHMGVERPGVTNVARSAMMKDSPFYQHYDLDLKDKPLGEGSFSICRKCVHKKSNQAFAVKIISKRMEANTQKEITALKLCEGHPNIVKLHEVFHDQLHTFLVMELLNGGELFERIKKKKHFSETEASYIMRKLVSAVSHMHDVGVVHRDLKPENLLFTDENDNLEIKIIDFGFARLKPPDNQPLKTPCFTLHYAAPELLNQNGYDESCDLWSLGVILYTMLSGQVPFQSHDRSLTCTSAVEIMKKIKKGDFSFEGEAWKNVSQEAKDLIQGLLTVDPNKRLKMSGLRYNEWLQDGSQLSSNPLMTPDILGSSGAAVHTCVKATFHAFNKYKREGFCLQNVDKAPLAKRRKMKKTSTSTETRSSSSESSHSSSSHSHGKTTPTKTLQPSNPADSNNPETLFQFSDSVA",
# KS6A4_HUMAN
"MGDEDDDESCAVELRITEANLTGHEEKVSVENFELLKVLGTGAYGKVFLVRKAGGHDAGKLYAMKVLRKAALVQRAKTQEHTRTERSVLELVRQAPFLVTLHYAFQTDAKLHLILDYVSGGEMFTHLYQRQYFKEAEVRVYGGEIVLALEHLHKLGIIYRDLKLENVLLDSEGHIVLTDFGLSKEFLTEEKERTFSFCGTIEYMAPEIIRSKTGHGKAVDWWSLGILLFELLTGASPFTLEGERNTQAEVSRRILKCSPPFPPRIGPVAQDLLQRLLCKDPKKRLGAGPQGAQEVRNHPFFQGLDWVALAARKIPAPFRPQIRSELDVGNFAEEFTRLEPVYSPPGSPPPGDPRIFQGYSFVAPSILFDHNNAVMTDGLEAPGAGDRPGRAAVARSAMMQDSPFFQQYELDLREPALGQGSFSVCRRCRQRQSGQEFAVKILSRRLEANTQREVAALRLCQSHPNVVNLHEVHHDQLHTYLVLELLRGGELLEHIRKKRHFSESEASQILRSLVSAVSFMHEEAGVVHRDLKPENILYADDTPGAPVKIIDFGFARLRPQSPGVPMQTPCFTLQYAAPELLAQQGYDESCDLWSLGVILYMMLSGQVPFQGASGQGGQSQAAEIMCKIREGRFSLDGEAWQGVSEEAKELVRGLLTVDPAKRLKLEGLRGSSWLQDGSARSSPPLRTPDVLESSGPAVRSGLNATFMAFNRGKREGFFLKSVENAPLAKRRKQKLRSATASRRGSPAPANPGRAPVASKGAPRRANGPLPPS",
# STK4_HUMAN
"METVQLRNPPRRQLKKLDEDSLTKQPEEVFDVLEKLGEGSYGSVYKAIHKETGQIVAIKQVPVESDLQEIIKEISIMQQCDSPHVVKYYGSYFKNTDLWIVMEYCGAGSVSDIIRLRNKTLTEDEIATILQSTLKGLEYLHFMRKIHRDIKAGNILLNTEGHAKLADFGVAGQLTDTMAKRNTVIGTPFWMAPEVIQEIGYNCVADIWSLGITAIEMAEGKPPYADIHPMRAIFMIPTNPPPTFRKPELWSDNFTDFVKQCLVKSPEQRATATQLLQHPFVRSAKGVSILRDLINEAMDVKLKRQESQQREVDQDDEENSEEDEMDSGTMVRAVGDEMGTVRVASTMTDGANTMIEHDDTLPSQLGTMVINAEDEEEEGTMKRRDETMQPAKPSFLEYFEQKEKENQINSFGKSVPGPLKNSSDWKIPQDGDYEFLKSWTVEDLQKRLLALDPMMEQEIEEIRQKYQSKRQPILDAIEAKKRRQQNF",
# STK3_HUMAN
"MEQPPAPKSKLKKLSEDSLTKQPEEVFDVLEKLGEGSYGSVFKAIHKESGQVVAIKQVPVESDLQEIIKEISIMQQCDSPYVVKYYGSYFKNTDLWIVMEYCGAGSVSDIIRLRNKTLIEDEIATILKSTLKGLEYLHFMRKIHRDIKAGNILLNTEGHAKLADFGVAGQLTDTMAKRNTVIGTPFWMAPEVIQEIGYNCVADIWSLGITSIEMAEGKPPYADIHPMRAIFMIPTNPPPTFRKPELWSDDFTDFVKKCLVKNPEQRATATQLLQHPFIKNAKPVSILRDLITEAMEIKAKRHEEQQRELEEEEENSDEDELDSHTMVKTSVESVGTMRATSTMSEGAQTMIEHNSTMLESDLGTMVINSEDEEEEDGTMKRNATSPQVQRPSFMDYFDKQDFKNKSHENCNQNMHEPFPMSKNVFPDNWKVPQDGDFDFLKNLSLEELQMRLKALDPMMEREIEELRQRYTAKRQPILDAMDAKKRRQQNF",
# STK24_HUMAN
"MDSRAQLWGLALNKRRATLPHPGGSTNLKADPEELFTKLEKIGKGSFGEVFKGIDNRTQKVVAIKIIDLEEAEDEIEDIQQEITVLSQCDSPYVTKYYGSYLKDTKLWIIMEYLGGGSALDLLEPGPLDETQIATILREILKGLDYLHSEKKIHRDIKAANVLLSEHGEVKLADFGVAGQLTDTQIKRNTFVGTPFWMAPEVIKQSAYDSKADIWSLGITAIELARGEPPHSELHPMKVLFLIPKNNPPTLEGNYSKPLKEFVEACLNKEPSFRPTAKELLKHKFILRNAKKTSYLTELIDRYKRWKAEQSHDDSSSEDSDAETDGQASGGSDSGDWIFTIREKDPKNLENGALQPSDLDRNKMKDIPKRPFSQCLSTIISPLFAELKEKSQACGGNLGSIEELRGAIYLAEEACPGISDTMVAQLVQRLQRYSLSGGGTSSH",
# MUSK_HUMAN
"MRELVNIPLVHILTLVAFSGTEKLPKAPVITTPLETVDALVEEVATFMCAVESYPQPEISWTRNKILIKLFDTRYSIRENGQLLTILSVEDSDDGIYCCTANNGVGGAVESCGALQVKMKPKITRPPINVKIIEGLKAVLPCTTMGNPKPSVSWIKGDSPLRENSRIAVLESGSLRIHNVQKEDAGQYRCVAKNSLGTAYSKVVKLEVEVFARILRAPESHNVTFGSFVTLHCTATGIPVPTITWIENGNAVSSGSIQESVKDRVIDSRLQLFITKPGLYTCIATNKHGEKFSTAKAAATISIAEWSKPQKDNKGYCAQYRGEVCNAVLAKDALVFLNTSYADPEEAQELLVHTAWNELKVVSPVCRPAAEALLCNHIFQECSPGVVPTPIPICREYCLAVKELFCAKEWLVMEEKTHRGLYRSEMHLLSVPECSKLPSMHWDPTACARLPHLDYNKENLKTFPPMTSSKPSVDIPNLPSSSSSSFSVSPTYSMTVIISIMSSFAIFVLLTITTLYCCRRRKQWKNKKRESAAVTLTTLPSELLLDRLHPNPMYQRMPLLLNPKLLSLEYPRNNIEYVRDIGEGAFGRVFQARAPGLLPYEPFTMVAVKMLKEEASADMQADFQREAALMAEFDNPNIVKLLGVCAVGKPMCLLFEYMAYGDLNEFLRSMSPHTVCSLSHSDLSMRAQVSSPGPPPLSCAEQLCIARQVAAGMAYLSERKFVHRDLATRNCLVGENMVVKIADFGLSRNIYSADYYKANENDAIPIRWMPPESIFYNRYTTESDVWAYGVVLWEIFSYGLQPYYGMAHEEVIYYVRDGNILSCPENCPVELYNLMRLCWSKLPADRPSFTSIHRILERMCERAEGTVSV",
# PMYT1_HUMAN
"MLERPPALAMPMPTEGTPPPLSGTPIPVPAYFRHAEPGFSLKRPRGLSRSLPPPPPAKGSIPISRLFPPRTPGWHQLQPRRVSFRGEASETLQSPGYDPSRPESFFQQSFQRLSRLGHGSYGEVFKVRSKEDGRLYAVKRSMSPFRGPKDRARKLAEVGSHEKVGQHPCCVRLEQAWEEGGILYLQTELCGPSLQQHCEAWGASLPEAQVWGYLRDTLLALAHLHSQGLVHLDVKPANIFLGPRGRCKLGDFGLLVELGTAGAGEVQEGDPRYMAPELLQGSYGTAADVFSLGLTILEVACNMELPHGGEGWQQLRQGYLPPEFTAGLSSELRSVLVMMLEPDPKLRATAEALLALPVLRQPRAWGVLWCMAAEALSRGWALWQALLALLCWLWHGLAHPASWLQPLGPPATPPGSPPCSLLLDSSLSSNWDDDSLGPSLSPEAVLARTVGSTSTPRSRCTPRDALDLSDINSEPPRGSFPSFEPRNLLSLFEDTLDPT",
# STK38_HUMAN
"MAMTGSTPCSSMSNHTKERVTMTKVTLENFYSNLIAQHEEREMRQKKLEKVMEEEGLKDEEKRLRRSAHARKETEFLRLKRTRLGLEDFESLKVIGRGAFGEVRLVQKKDTGHVYAMKILRKADMLEKEQVGHIRAERDILVEADSLWVVKMFYSFQDKLNLYLIMEFLPGGDMMTLLMKKDTLTEEETQFYIAETVLAIDSIHQLGFIHRDIKPDNLLLDSKGHVKLSDFGLCTGLKKAHRTEFYRNLNHSLPSDFTFQNMNSKRKAETWKRNRRQLAFSTVGTPDYIAPEVFMQTGYNKLCDWWSLGVIMYEMLIGYPPFCSETPQETYKKVMNWKETLTFPPEVPISEKAKDLILRFCCEWEHRIGAPGVEEIKSNSFFEGVDWEHIRERPAAISIEIKSIDDTSNFDEFPESDILKPTVATSNHPETDYKNKDWVFINYTYKRFEGLTARGAIPSYMKAAK",
# NEK1_HUMAN
"MEKYVRLQKIGEGSFGKAILVKSTEDGRQYVIKEINISRMSSKEREESRREVAVLANMKHPNIVQYRESFEENGSLYIVMDYCEGGDLFKRINAQKGVLFQEDQILDWFVQICLALKHVHDRKILHRDIKSQNIFLTKDGTVQLGDFGIARVLNSTVELARTCIGTPYYLSPEICENKPYNNKSDIWALGCVLYELCTLKHAFEAGSMKNLVLKIISGSFPPVSLHYSYDLRSLVSQLFKRNPRDRPSVNSILEKGFIAKRIEKFLSPQLIAEEFCLKTFSKFGSQPIPAKRPASGQNSISVMPAQKITKPAAKYGIPLAYKKYGDKKLHEKKPLQKHKQAHQTPEKRVNTGEERRKISEEAARKRRLEFIEKEKKQKDQIISLMKAEQMKRQEKERLERINRAREQGWRNVLSAGGSGEVKAPFLGSGGTIAPSSFSSRGQYEHYHAIFDQMQQQRAEDNEAKWKREIYGRGLPERGILPGVRPGFPYGAAGHHHFPDADDIRKTLKRLKAVSKQANANRQKGQLAVERAKQVEEFLQRKREAMQNKARAEGHMVYLARLRQIRLQNFNERQQIKAKLRGEKKEANHSEGQEGSEEADMRRKKIESLKAHANARAAVLKEQLERKRKEAYEREKKVWEEHLVAKGVKSSDVSPPLGQHETGGSPSKQQMRSVISVTSALKEVGVDSSLTDTRETSEEMQKTNNAISSKREILRRLNENLKAQEDEKGKQNLSDTFEINVHEDAKEHEKEKSVSSDRKKWEAGGQLVIPLDELTLDTSFSTTERHTVGEVIKLGPNGSPRRAWGKSPTDSVLKILGEAELQLQTELLENTTIRSEISPEGEKYKPLITGEKKVQCISHEINPSAIVDSPVETKSPEFSEASPQMSLKLEGNLEEPDDLETEILQEPSGTNKDESLPCTITDVWISEEKETKETQSADRITIQENEVSEDGVSSTVDQLSDIHIEPGTNDSQHSKCDVDKSVQPEPFFHKVVHSEHLNLVPQVQSVQCSPEESFAFRSHSHLPPKNKNKNSLLIGLSTGLFDANNPKMLRTCSLPDLSKLFRTLMDVPTVGDVRQDNLEIDEIEDENIKEGPSDSEDIVFEETDTDLQELQASMEQLLREQPGEEYSEEEESVLKNSDVEPTANGTDVADEDDNPSSESALNEEWHSDNSDGEIASECECDSVFNHLEELRLHLEQEMGFEKFFEVYEKIKAIHEDEDENIEICSKIVQNILGNEHQHLYAKILHLVMADGAYQEDNDE",
# NEK2_HUMAN
"MPSRAEDYEVLYTIGTGSYGRCQKIRRKSDGKILVWKELDYGSMTEAEKQMLVSEVNLLRELKHPNIVRYYDRIIDRTNTTLYIVMEYCEGGDLASVITKGTKERQYLDEEFVLRVMTQLTLALKECHRRSDGGHTVLHRDLKPANVFLDGKQNVKLGDFGLARILNHDTSFAKTFVGTPYYMSPEQMNRMSYNEKSDIWSLGCLLYELCALMPPFTAFSQKELAGKIREGKFRRIPYRYSDELNEIITRMLNLKDYHRPSVEEILENPLIADLVADEQRRNLERRGRQLGEPEKSQDSSPVLSELKLKEIQLQERERALKAREERLEQKEQELCVRERLAEDKLARAENLLKNYSLLKERKFLSLASNPELLNLPSSVIKKKVHFSGESKENIMRSENSESQLTSKSKCKDLKKRLHAAQLRAQALSDIEKNYQLKSRQILGMR",
# NEK3_HUMAN
"MDDYMVLRMIGEGSFGRALLVQHESSNQMFAMKEIRLPKSFSNTQNSRKEAVLLAKMKHPNIVAFKESFEAEGHLYIVMEYCDGGDLMQKIKQQKGKLFPEDMILNWFTQMCLGVNHIHKKRVLHRDIKSKNIFLTQNGKVKLGDFGSARLLSNPMAFACTYVGTPYYVPPEIWENLPYNNKSDIWSLGCILYELCTLKHPFQANSWKNLILKVCQGCISPLPSHYSYELQFLVKQMFKRNPSHRPSATTLLSRGIVARLVQKCLPPEIIMEYGEEVLEEIKNSKHNTPRKKTNPSRIRIALGNEASTVQEEEQDRKGSHTDLESINENLVESALRRVNREEKGNKSVHLRKASSPNLHRRQWEKNVPNTALTALENASILTSSLTAEDDRGGSVIKYSKNTTRKQWLKETPDTLLNILKNADLSLAFQTYTIYRPGSEGFLKGPLSEETEASDSVDGGHDSVILDPERLEPGLDEEDTDFEEEDDNPDWVSELKKRAGWQGLCDR",
# M3K14_HUMAN
"MAVMEMACPGAPGSAVGQQKELPKAKEKTPPLGKKQSSVYKLEAVEKSPVFCGKWEILNDVITKGTAKEGSEAGPAAISIIAQAECENSQEFSPTFSERIFIAGSKQYSQSESLDQIPNNVAHATEGKMARVCWKGKRRSKARKKRKKKSSKSLAHAGVALAKPLPRTPEQESCTIPVQEDESPLGAPYVRNTPQFTKPLKEPGLGQLCFKQLGEGLRPALPRSELHKLISPLQCLNHVWKLHHPQDGGPLPLPTHPFPYSRLPHPFPFHPLQPWKPHPLESFLGKLACVDSQKPLPDPHLSKLACVDSPKPLPGPHLEPSCLSRGAHEKFSVEEYLVHALQGSVSSGQAHSLTSLAKTWAARGSRSREPSPKTEDNEGVLLTEKLKPVDYEYREEVHWATHQLRLGRGSFGEVHRMEDKQTGFQCAVKKVRLEVFRAEELMACAGLTSPRIVPLYGAVREGPWVNIFMELLEGGSLGQLVKEQGCLPEDRALYYLGQALEGLEYLHSRRILHGDVKADNVLLSSDGSHAALCDFGHAVCLQPDGLGKSLLTGDYIPGTETHMAPEVVLGRSCDAKVDVWSSCCMMLHMLNGCHPWTQFFRGPLCLKIASEPPPVREIPPSCAPLTAQAIQEGLRKEPIHRVSAAELGGKVNRALQQVGGLKSPWRGEYKEPRHPPPNQANYHQTLHAQPRELSPRAPGPRPAEETTGRAPKLQPPLPPEPPEPNKSPPLTLSKEESGMWEPLPLSSLEPAPARNPSSPERKATVPEQELQQLEIELFLNSLSQPFSLEEQEQILSCLSIDSLSLSDDSEKNPSKASQSSRDTLSSGVHSWSSQAEARSSSWNMVLARGRPTDTPSYFNGVKVQIQSLNGEHLHIREFHRVKVGDIATGISSQIPAAAFSLVTKDGQPVRYDMEVPDSGIDLQCTLAPDGSFAWSWRVKHGQLENRP",
# NLK_HUMAN
"MSLCGARANAKMMAAYNGGTSAAAAGHHHHHHHHLPHLPPPHLHHHHHPQHHLHPGSAAAVHPVQQHTSSAAAAAAAAAAAAAMLNPGQQQPYFPSPAPGQAPGPAAAAPAQVQAAAAATVKAHHHQHSHHPQQQLDIEPDRPIGYGAFGVVWSVTDPRDGKRVALKKMPNVFQNLVSCKRVFRELKMLCFFKHDNVLSALDILQPPHIDYFEEIYVVTELMQSDLHKIIVSPQPLSSDHVKVFLYQILRGLKYLHSAGILHRDIKPGNLLVNSNCVLKICDFGLARVEELDESRHMTQEVVTQYYRAPEILMGSRHYSNAIDIWSVGCIFAELLGRRILFQAQSPIQQLDLITDLLGTPSLEAMRTACEGAKAHILRGPHKQPSLPVLYTLSSQATHEAVHLLCRMLVFDPSKRISAKDALAHPYLDEGRLRYHTCMCKCCFSTSTGRVYTSDFEPVTNPKFDDTFEKNLSSVRQVKEIIHQFILEQQKGNRVPLCINPQSAAFKSFISSTVAQPSEMPPSPLVWE",
# NEK4_HUMAN
"MPLAAYCYLRVVGKGSYGEVTLVKHRRDGKQYVIKKLNLRNASSRERRAAEQEAQLLSQLKHPNIVTYKESWEGGDGLLYIVMGFCEGGDLYRKLKEQKGQLLPENQVVEWFVQIAMALQYLHEKHILHRDLKTQNVFLTRTNIIKVGDLGIARVLENHCDMASTLIGTPYYMSPELFSNKPYNYKSDVWALGCCVYEMATLKHAFNAKDMNSLVYRIIEGKLPPMPRDYSPELAELIRTMLSKRPEERPSVRSILRQPYIKRQISFFLEATKIKTSKNNIKNGDSQSKPFATVVSGEAESNHEVIHPQPLSSEGSQTYIMGEGKCLSQEKPRASGLLKSPASLKAHTCKQDLSNTTELATISSVNIDILPAKGRDSVSDGFVQENQPRYLDASNELGGICSISQVEEEMLQDNTKSSAQPENLIPMWSSDIVTGEKNEPVKPLQPLIKEQKPKDQSLALSPKLECSGTILAHSNLRLLGSSDSPASASRVAGITGVCHHAQDQVAGECIIEKQGRIHPDLQPHNSGSEPSLSRQRRQKRREQTEHRGEKRQVRRDLFAFQESPPRFLPSHPIVGKVDVTSTQKEAENQRRVVTGSVSSSRSSEMSSSKDRPLSARERRRLKQSQEEMSSSGPSVRKASLSVAGPGKPQEEDQPLPARRLSSDCSVTQERKQIHCLSEDELSSSTSSTDKSDGDYGEGKGQTNEINALVQLMTQTLKLDSKESCEDVPVANPVSEFKLHRKYRDTLILHGKVAEEAEEIHFKELPSAIMPGSEKIRRLVEVLRTDVIRGLGVQLLEQVYDLLEEEDEFDREVRLREHMGEKYTTYSVKARQLKFFEENMNF",
# IRAK4_HUMAN
"MNKPITPSTYVRCLNVGLIRKLSDFIDPQEGWKKLAVAIKKPSGDDRYNQFHIRRFEALLQTGKSPTSELLFDWGTTNCTVGDLVDLLIQNEFFAPASLLLPDAVPKTANTLPSKEAITVQQKQMPFCDKDRTLMTPVQNLEQSYMPPDSSSPENKSLEVSDTRFHSFSFYELKNVTNNFDERPISVGGNKMGEGGFGVVYKGYVNNTTVAVKKLAAMVDITTEELKQQFDQEIKVMAKCQHENLVELLGFSSDGDDLCLVYVYMPNGSLLDRLSCLDGTPPLSWHMRCKIAQGAANGINFLHENHHIHRDIKSANILLDEAFTAKISDFGLARASEKFAQTVMTSRIVGTTAYMAPEALRGEITPKSDIYSFGVVLLEIITGLPAVDEHREPQLLLDIKEEIEDEEKTIEDYIDKKMNDADSTSVEAMYSVASQCLHEKKNKRPDIKKVQQLLQEMTAS",
# PI3R4_HUMAN
"MGNQLAGIAPSQILSVESYFSDIHDFEYDKSLGSTRFFKVARAKHREGLVVVKVFAIQDPTLPLTSYKQELEELKIRLNSAQNCLPFQKASEKASEKAAMLFRQYVRDNLYDRISTRPFLNNIEKRWIAFQILTAVDQAHKSGVRHGDIKTENVMVTSWNWVLLTDFASFKPTYLPEDNPADFNYFFDTSRRRTCYIAPERFVDGGMFATELEYMRDPSTPLVDLNSNQRTRGELKRAMDIFSAGCVIAELFTEGVPLFDLSQLLAYRNGHFFPEQVLNKIEDHSIRELVTQMIHREPDKRLEAEDYLKQQRGNAFPEIFYTFLQPYMAQFAKETFLSADERILVIRKDLGNIIHNLCGHDLPEKAEGEPKENGLVILVSVITSCLQTLKYCDSKLAALELILHLAPRLSVEILLDRITPYLLHFSNDSVPRVRAEALRTLTKVLALVKEVPRNDINIYPEYILPGIAHLAQDDATIVRLAYAENIALLAETALRFLELVQLKNLNMENDPNNEEIDEVTHPNGNYDTELQALHEMVQQKVVTLLSDPENIVKQTLMENGITRLCVFFGRQKANDVLLSHMITFLNDKNDWHLRGAFFDSIVGVAAYVGWQSSSILKPLLQQGLSDAEEFVIVKALYALTCMCQLGLLQKPHVYEFASDIAPFLCHPNLWIRYGAVGFITVVARQISTADVYCKLMPYLDPYITQPIIQIERKLVLLSVLKEPVSRSIFDYALRSKDITSLFRHLHMRQKKRNGSLPDCPPPEDPAIAQLLKKLLSQGMTEEEEDKLLALKDFMMKSNKAKANIVDQSHLHDSSQKGVIDLAALGITGRQVDLVKTKQEPDDKRARKHVKQDSNVNEEWKSMFGSLDPPNMPQALPKGSDQEVIQTGKPPRSESSAGICVPLSTSSQVPEVTTVQNKKPVIPVLSSTILPSTYQIRITTCKTELQQLIQQKREQCNAERIAKQMMENAEWESKPPPPGWRPKGLLVAHLHEHKSAVNRIRVSDEHSLFATCSNDGTVKIWNSQKMEGKTTTTRSILTYSRIGGRVKTLTFCQGSHYLAIASDNGAVQLLGIEASKLPKSPKIHPLQSRILDQKEDGCVVDMHHFNSGAQSVLAYATVNGSLVGWDLRSSSNAWTLKHDLKSGLITSFAVDIHQCWLCIGTSSGTMACWDMRFQLPISSHCHPSRARIRRLSMHPLYQSWVIAAVQGNNEVSMWDMETGDRRFTLWASSAPPLSELQPSPHSVHGIYCSPADGNPILLTAGSDMKIRFWDLAYPERSYVVAGSTSSPSVSYYRKIIEGTEVVQEIQNKQKVGPSDDTPRRGPESLPVGHHDIITDVATFQTTQGFIVTASRDGIVKVWK",
# ROCK2_HUMAN
"MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESLLDGLNSLVLDLDFPALRKNKNIDNFLNRYEKIVKKIRGLQMKAEDYDVVKVIGRGAFGEVQLVRHKASQKVYAMKLLSKFEMIKRSDSAFFWEERDIMAFANSPWVVQLFYAFQDDRYLYMVMEYMPGGDLVNLMSNYDVPEKWAKFYTAEVVLALDAIHSMGLIHRDVKPDNMLLDKHGHLKLADFGTCMKMDETGMVHCDTAVGTPDYISPEVLKSQGGDGFYGRECDWWSVGVFLYEMLVGDTPFYADSLVGTYSKIMDHKNSLCFPEDAEISKHAKNLICAFLTDREVRLGRNGVEEIRQHPFFKNDQWHWDNIRETAAPVVPELSSDIDSSNFDDIEDDKGDVETFPIPKAFVGNQLPFIGFTYYRENLLLSDSPSCRETDSIQSRKNEESQEIQKKLYTLEEHLSNEMQAKEELEQKCKSVNTRLEKTAKELEEEITLRKSVESALRQLEREKALLQHKNAEYQRKADHEADKKRNLENDVNSLKDQLEDLKKRNQNSQISTEKVNQLQRQLDETNALLRTESDTAARLRKTQAESSKQIQQLESNNRDLQDKNCLLETAKLKLEKEFINLQSALESERRDRTHGSEIINDLQGRICGLEEDLKNGKILLAKVELEKRQLQERFTDLEKEKSNMEIDMTYQLKVIQQSLEQEEAEHKATKARLADKNKIYESIEEAKSEAMKEMEKKLLEERTLKQKVENLLLEAEKRCSLLDCDLKQSQQKINELLKQKDVLNEDVRNLTLKIEQETQKRCLTQNDLKMQTQQVNTLKMSEKQLKQENNHLMEMKMNLEKQNAELRKERQDADGQMKELQDQLEAEQYFSTLYKTQVRELKEECEEKTKLGKELQQKKQELQDERDSLAAQLEITLTKADSEQLARSIAEEQYSDLEKEKIMKELEIKEMMARHKQELTEKDATIASLEETNRTLTSDVANLANEKEELNNKLKDVQEQLSRLKDEEISAAAIKAQFEKQLLTERTLKTQAVNKLAEIMNRKEPVKRGNDTDVRRKEKENRKLHMELKSEREKLTQQMIKYQKELNEMQAQIAEESQIRIELQMTLDSKDSDIEQLRSQLQALHIGLDSSSIGSGPGDAEADDGFPESRLEGWLSLPVRNNTKKFGWVKKYVIVSSKKILFYDSEQDKEQSNPYMVLDIDKLFHVRPVTQTDVYRADAKEIPRIFQILYANEGESKKEQEFPVEPVGEKSNYICHKGHEFIPTLYHFPTNCEACMKPLWHMFKPPPALECRRCHIKCHKDHMDKKEEIIAPCKVYYDISTAKNLLLLANSTEEQQKWVSRLVKKIPKKPPAPDPFARSSPRTSMKIQQNQSIRRPSRQLAPNKPS",
# MK14_HUMAN
"MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKTGLRVAVKKLSRPFQSIIHAKRTYRELRLLKHMKHENVIGLLDVFTPARSLEEFNDVYLVTHLMGADLNNIVKCQKLTDDHVQFLIYQILRGLKYIHSADIIHRDLKPSNLAVNEDCELKILDFGLARHTDDEMTGYVATRWYRAPEIMLNWMHYNQTVDIWSVGCIMAELLTGRTLFPGTDHIDQLKLILRLVGTPGAELLKKISSESARNYIQSLTQMPKMNFANVFIGANPLAVDLLEKMLVLDSDKRITAAQALAHAYFAQYHDPDDEPVADPYDQSFESRDLLIDEWKSLTYDEVISFVPPPLDQEEMES",
# KS6B1_HUMAN
"MRRRRRRDGFYPAPDFRDREAEDMAGVFDIDLDQPEDAGSEDELEEGGQLNESMDHGGVGPYELGMEHCEKFEISETSVNRGPEKIRPECFELLRVLGKGGYGKVFQVRKVTGANTGKIFAMKVLKKAMIVRNAKDTAHTKAERNILEEVKHPFIVDLIYAFQTGGKLYLILEYLSGGELFMQLEREGIFMEDTACFYLAEISMALGHLHQKGIIYRDLKPENIMLNHQGHVKLTDFGLCKESIHDGTVTHTFCGTIEYMAPEILMRSGHNRAVDWWSLGALMYDMLTGAPPFTGENRKKTIDKILKCKLNLPPYLTQEARDLLKKLLKRNAASRLGAGPGDAGEVQAHPFFRHINWEELLARKVEPPFKPLLQSEEDVSQFDSKFTRQTPVDSPDDSTLSESANQVFLGFTYVAPSVLESVKEKFSFEPKIRSPRRFIGSPRTPVSPVKFSPGDFWGRGASASTANPQTPVEYPMETSGIEQMDVTMSGEASAPLPIRQPNSGPYKKQAFPMISKRPEHLRMNL",
# KS6B2_HUMAN
"MAAVFDLDLETEEGSEGEGEPELSPADACPLAELRAAGLEPVGHYEEVELTETSVNVGPERIGPHCFELLRVLGKGGYGKVFQVRKVQGTNLGKIYAMKVLRKAKIVRNAKDTAHTRAERNILESVKHPFIVELAYAFQTGGKLYLILECLSGGELFTHLEREGIFLEDTACFYLAEITLALGHLHSQGIIYRDLKPENIMLSSQGHIKLTDFGLCKESIHEGAVTHTFCGTIEYMAPEILVRSGHNRAVDWWSLGALMYDMLTGSPPFTAENRKKTMDKIIRGKLALPPYLTPDARDLVKKFLKRNPSQRIGGGPGDAADVQRHPFFRHMNWDDLLAWRVDPPFRPCLQSEEDVSQFDTRFTRQTPVDSPDDTALSESANQAFLGFTYVAPSVLDSIKEGFSFQPKLRSPRRLNSSPRAPVSPLKFSPFEGFRPSPSLPEPTELPLPPLLPPPPPSTTAPLPIRPPSGTKKSKRGRGRPGR",
# PAK1_HUMAN
"MSNNGLDIQDKPPAPPMRNTSTMIGAGSKDAGTLNHGSKPLPPNPEEKKKKDRFYRSILPGDKTNKKKEKERPEISLPSDFEHTIHVGFDAVTGEFTGMPEQWARLLQTSNITKSEQKKNPQAVLDVLEFYNSKKTSNSQKYMSFTDKSAEDYNSSNALNVKAVSETPAVPPVSEDEDDDDDDATPPPVIAPRPEHTKSVYTRSVIEPLPVTPTRDVATSPISPTENNTTPPDALTRNTEKQKKKPKMSDEEILEKLRSIVSVGDPKKKYTRFEKIGQGASGTVYTAMDVATGQEVAIKQMNLQQQPKKELIINEILVMRENKNPNIVNYLDSYLVGDELWVVMEYLAGGSLTDVVTETCMDEGQIAAVCRECLQALEFLHSNQVIHRDIKSDNILLGMDGSVKLTDFGFCAQITPEQSKRSTMVGTPYWMAPEVVTRKAYGPKVDIWSLGIMAIEMIEGEPPYLNENPLRALYLIATNGTPELQNPEKLSAIFRDFLNRCLEMDVEKRGSAKELLQHQFLKIAKPLSSLTPLIAAAKEATKNNH",
# PAK2_HUMAN
"MSDNGELEDKPPAPPVRMSSTIFSTGGKDPLSANHSLKPLPSVPEEKKPRHKIISIFSGTEKGSKKKEKERPEISPPSDFEHTIHVGFDAVTGEFTGMPEQWARLLQTSNITKLEQKKNPQAVLDVLKFYDSNTVKQKYLSFTPPEKDGFPSGTPALNAKGTEAPAVVTEEEDDDEETAPPVIAPRPDHTKSIYTRSVIDPVPAPVGDSHVDGAAKSLDKQKKKTKMTDEEIMEKLRTIVSIGDPKKKYTRYEKIGQGASGTVFTATDVALGQEVAIKQINLQKQPKKELIINEILVMKELKNPNIVNFLDSYLVGDELFVVMEYLAGGSLTDVVTETCMDEAQIAAVCRECLQALEFLHANQVIHRDIKSDNVLLGMEGSVKLTDFGFCAQITPEQSKRSTMVGTPYWMAPEVVTRKAYGPKVDIWSLGIMAIEMVEGEPPYLNENPLRALYLIATNGTPELQNPEKLSPIFRDFLNRCLEMDVEKRGSAKELLQHPFLKLAKPLSSLTPLIMAAKEAMKSNR",
# PAK3_HUMAN
"MSDGLDNEEKPPAPPLRMNSNNRDSSALNHSSKPLPMAPEEKNKKARLRSIFPGGGDKTNKKKEKERPEISLPSDFEHTIHVGFDAVTGEFTPDLYGSQMCPGKLPEGIPEQWARLLQTSNITKLEQKKNPQAVLDVLKFYDSKETVNNQKYMSFTSGDKSAHGYIAAHPSSTKTASEPPLAPPVSEEEDEEEEEEEDENEPPPVIAPRPEHTKSIYTRSVVESIASPAVPNKEVTPPSAENANSSTLYRNTDRQRKKSKMTDEEILEKLRSIVSVGDPKKKYTRFEKIGQGASGTVYTALDIATGQEVAIKQMNLQQQPKKELIINEILVMRENKNPNIVNYLDSYLVGDELWVVMEYLAGGSLTDVVTETCMDEGQIAAVCRECLQALDFLHSNQVIHRDIKSDNILLGMDGSVKLTDFGFCAQITPEQSKRSTMVGTPYWMAPEVVTRKAYGPKVDIWSLGIMAIEMVEGEPPYLNENPLRALYLIATNGTPELQNPERLSAVFRDFLNRCLEMDVDRRGSAKELLQHPFLKLAKPLSSLTPLIIAAKEAIKNSSR",
# KPCL_HUMAN
"MSSGTMKFNGYLRVRIGEAVGLQPTRWSLRHSLFKKGHQLLDPYLTVSVDQVRVGQTSTKQKTNKPTYNEEFCANVTDGGHLELAVFHETPLGYDHFVANCTLQFQELLRTTGASDTFEGWVDLEPEGKVFVVITLTGSFTEATLQRDRIFKHFTRKRQRAMRRRVHQINGHKFMATYLRQPTYCSHCREFIWGVFGKQGYQCQVCTCVVHKRCHHLIVTACTCQNNINKVDSKIAEQRFGINIPHKFSIHNYKVPTFCDHCGSLLWGIMRQGLQCKICKMNVHIRCQANVAPNCGVNAVELAKTLAGMGLQPGNISPTSKLVSRSTLRRQGKESSKEGNGIGVNSSNRLGIDNFEFIRVLGKGSFGKVMLARVKETGDLYAVKVLKKDVILQDDDVECTMTEKRILSLARNHPFLTQLFCCFQTPDRLFFVMEFVNGGDLMFHIQKSRRFDEARARFYAAEIISALMFLHDKGIIYRDLKLDNVLLDHEGHCKLADFGMCKEGICNGVTTATFCGTPDYIAPEILQEMLYGPAVDWWAMGVLLYEMLCGHAPFEAENEDDLFEAILNDEVVYPTWLHEDATGILKSFMTKNPTMRLGSLTQGGEHAILRHPFFKEIDWAQLNHRQIEPPFRPRIKSREDVSNFDPDFIKEEPVLTPIDEGHLPMINQDEFRNFSYVSPELQP",
# CDK16_HUMAN
"MDRMKKIKRQLSMTLRGGRGIDKTNGAPEQIGLDESGGGGGSDPGEAPTRAAPGELRSARGPLSSAPEIVHEDLKMGSDGESDQASATSSDEVQSPVRVRMRNHPPRKISTEDINKRLSLPADIRLPEGYLEKLTLNSPIFDKPLSRRLRRVSLSEIGFGKLETYIKLDKLGEGTYATVYKGKSKLTDNLVALKEIRLEHEEGAPCTAIREVSLLKDLKHANIVTLHDIIHTEKSLTLVFEYLDKDLKQYLDDCGNIINMHNVKLFLFQLLRGLAYCHRQKVLHRDLKPQNLLINERGELKLADFGLARAKSIPTKTYSNEVVTLWYRPPDILLGSTDYSTQIDMWGVGCIFYEMATGRPLFPGSTVEEQLHFIFRILGTPTEETWPGILSNEEFKTYNYPKYRAEALLSHAPRLDSDGADLLTKLLQFEGRNRISAEDAMKHPFFLSLGERIHKLPDTTSIFALKEIQLQKEASLRSSSMPDSGRPAFRVVDTEF",
# CDK17_HUMAN
"MKKFKRRLSLTLRGSQTIDESLSELAEQMTIEENSSKDNEPIVKNGRPPTSHSMHSFLHQYTGSFKKPPLRRPHSVIGGSLGSFMAMPRNGSRLDIVHENLKMGSDGESDQASGTSSDEVQSPTGVCLRNRIHRRISMEDLNKRLSLPADIRIPDGYLEKLQINSPPFDQPMSRRSRRASLSEIGFGKMETYIKLEKLGEGTYATVYKGRSKLTENLVALKEIRLEHEEGAPCTAIREVSLLKDLKHANIVTLHDIVHTDKSLTLVFEYLDKDLKQYMDDCGNIMSMHNVKLFLYQILRGLAYCHRRKVLHRDLKPQNLLINEKGELKLADFGLARAKSVPTKTYSNEVVTLWYRPPDVLLGSSEYSTQIDMWGVGCIFFEMASGRPLFPGSTVEDELHLIFRLLGTPSQETWPGISSNEEFKNYNFPKYKPQPLINHAPRLDSEGIELITKFLQYESKKRVSAEEAMKHVYFRSLGPRIHALPESVSIFSLKEIQLQKDPGFRNSSYPETGHGKNRRQSMLF",
# CDK18_HUMAN
"MIMNKMKNFKRRFSLSVPRTETIEESLAEFTEQFNQLHNRRNENLQLGPLGRDPPQECSTFSPTDSGEEPGQLSPGVQFQRRQNQRRFSMEDVSKRLSLPMDIRLPQEFLQKLQMESPDLPKPLSRMSRRASLSDIGFGKLETYVKLDKLGEGTYATVFKGRSKLTENLVALKEIRLEHEEGAPCTAIREVSLLKNLKHANIVTLHDLIHTDRSLTLVFEYLDSDLKQYLDHCGNLMSMHNVKIFMFQLLRGLAYCHHRKILHRDLKPQNLLINERGELKLADFGLARAKSVPTKTYSNEVVTLWYRPPDVLLGSTEYSTPIDMWGVGCIHYEMATGRPLFPGSTVKEELHLIFRLLGTPTEETWPGVTAFSEFRTYSFPCYLPQPLINHAPRLDTDGIHLLSSLLLYESKSRMSAEAALSHSYFRSLGERVHQLEDTASIFSLKEIQLQKDPGYRGLAFQQPGRGKNRRQSIF",
# PGFRA_HUMAN
"MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFSLRCFGESEVSWQYPMSEEESSDVEIRNEENNSGLFVTVLEVSSASAAHTGLYTCYYNHTQTEENELEGRHIYIYVPDPDVAFVPLGMTDYLVIVEDDDSAIIPCRTTDPETPVTLHNSEGVVPASYDSRQGFNGTFTVGPYICEATVKGKKFQTIPFNVYALKATSELDLEMEALKTVYKSGETIVVTCAVFNNEVVDLQWTYPGEVKGKGITMLEEIKVPSIKLVYTLTVPEATVKDSGDYECAARQATREVKEMKKVTISVHEKGFIEIKPTFSQLEAVNLHEVKHFVVEVRAYPPPRISWLKNNLTLIENLTEITTDVEKIQEIRYRSKLKLIRAKEEDSGHYTIVAQNEDAVKSYTFELLTQVPSSILDLVDDHHGSTGGQTVRCTAEGTPLPDIEWMICKDIKKCNNETSWTILANNVSNIITEIHSRDRSTVEGRVTFAKVEETIAVRCLAKNLLGAENRELKLVAPTLRSELTVAAAVLVLLVIVIISLIVLVVIWKQKPRYEIRWRVIESISPDGHEYIYVDPMQLPYDSRWEFPRDGLVLGRVLGSGAFGKVVEGTAYGLSRSQPVMKVAVKMLKPTARSSEKQALMSELKIMTHLGPHLNIVNLLGACTKSGPIYIITEYCFYGDLVNYLHKNRDSFLSHHPEKPKKELDIFGLNPADESTRSYVILSFENNGDYMDMKQADTTQYVPMLERKEVSKYSDIQRSLYDRPASYKKKSMLDSEVKNLLSDDNSEGLTLLDLLSFTYQVARGMEFLASKNCVHRDLAARNVLLAQGKIVKICDFGLARDIMHDSNYVSKGSTFLPVKWMAPESIFDNLYTTLSDVWSYGILLWEIFSLGGTPYPGMMVDSTFYNKIKSGYRMAKPDHATSEVYEIMVKCWNSEPEKRPSFYHLSEIVENLLPGQYKKSYEKIHLDFLKSDHPAVARMRVDSDNAYIGVTYKNEEDKLKDWEGGLDEQRLSADSGYIIPLPDIDPVPEEEDLGKRNRHSSQTSEESAIETGSSSSTFIKREDETIEDIDMMDDIGIDSSDLVEDSFL",
# PGFRB_HUMAN
"MRLPGAMPALALKGELLLLSLLLLLEPQISQGLVVTPPGPELVLNVSSTFVLTCSGSAPVVWERMSQEPPQEMAKAQDGTFSSVLTLTNLTGLDTGEYFCTHNDSRGLETDERKRLYIFVPDPTVGFLPNDAEELFIFLTEITEITIPCRVTDPQLVVTLHEKKGDVALPVPYDHQRGFSGIFEDRSYICKTTIGDREVDSDAYYVYRLQVSSINVSVNAVQTVVRQGENITLMCIVIGNEVVNFEWTYPRKESGRLVEPVTDFLLDMPYHIRSILHIPSAELEDSGTYTCNVTESVNDHQDEKAINITVVESGYVRLLGEVGTLQFAELHRSRTLQVVFEAYPPPTVLWFKDNRTLGDSSAGEIALSTRNVSETRYVSELTLVRVKVAEAGHYTMRAFHEDAEVQLSFQLQINVPVRVLELSESHPDSGEQTVRCRGRGMPQPNIIWSACRDLKRCPRELPPTLLGNSSEEESQLETNVTYWEEEQEFEVVSTLRLQHVDRPLSVRCTLRNAVGQDTQEVIVVPHSLPFKVVVISAILALVVLTIISLIILIMLWQKKPRYEIRWKVIESVSSDGHEYIYVDPMQLPYDSTWELPRDQLVLGRTLGSGAFGQVVEATAHGLSHSQATMKVAVKMLKSTARSSEKQALMSELKIMSHLGPHLNVVNLLGACTKGGPIYIITEYCRYGDLVDYLHRNKHTFLQHHSDKRRPPSAELYSNALPVGLPLPSHVSLTGESDGGYMDMSKDESVDYVPMLDMKGDVKYADIESSNYMAPYDNYVPSAPERTCRATLINESPVLSYMDLVGFSYQVANGMEFLASKNCVHRDLAARNVLICEGKLVKICDFGLARDIMRDSNYISKGSTFLPLKWMAPESIFNSLYTTLSDVWSFGILLWEIFTLGGTPYPELPMNEQFYNAIKRGYRMAQPAHASDEIYEIMQKCWEEKFEIRPPFSQLVLLLERLLGEGYKKKYQQVDEEFLRSDHPAILRSQARLPGFHGLRSPLDTSSVLYTAVQPNEGDNDYIIPLPDPKPEVADEGPLEGSPSLASSTLNEVNTSSTISCDSPLEPQDEPEPEPQLELQVEPEPELEQLPDSGCPAPRAEAEDSFL",
# PDPK1_HUMAN
"MARTTSQLYDAVPIQSSVVLCSCPSPSMVRTQTESSTPPGIPGGSRQGPAMDGTAAEPRPGAGSLQHAQPPPQPRKKRPEDFKFGKILGEGSFSTVVLARELATSREYAIKILEKRHIIKENKVPYVTRERDVMSRLDHPFFVKLYFTFQDDEKLYFGLSYAKNGELLKYIRKIGSFDETCTRFYTAEIVSALEYLHGKGIIHRDLKPENILLNEDMHIQITDFGTAKVLSPESKQARANSFVGTAQYVSPELLTEKSACKSSDLWALGCIIYQLVAGLPPFRAGNEYLIFQKIIKLEYDFPEKFFPKARDLVEKLLVLDATKRLGCEEMEGYGPLKAHPFFESVTWENLHQQTPPKLTAYLPAMSEDDEDCYGNYDNLLSQFGCMQVSSSSSSHSLSASDTGLPQRSGSNIEQYIHDLDSNSFELDLQFSEDEKRLLLEKQAGGNPWHQFVENNLILKMGPVDKRKGLFARRRQLLLTEGPHLYYVDPVNKVLKGEIPWSQELRPEAKNFKTFFVHTPNRTYYLMDPSGNAHKWCRKIQEVWRQRYQSHPDAAVQ",
# PDK1_HUMAN
"MRLARLLRGAALAGPGPGLRAAGFSRSFSSDSGSSPASERGVPGQVDFYARFSPSPLSMKQFLDFGSVNACEKTSFMFLRQELPVRLANIMKEISLLPDNLLRTPSVQLVQSWYIQSLQELLDFKDKSAEDAKAIYDFTDTVIRIRNRHNDVIPTMAQGVIEYKESFGVDPVTSQNVQYFLDRFYMSRISIRMLLNQHSLLFGGKGKGSPSHRKHIGSINPNCNVLEVIKDGYENARRLCDLYYINSPELELEELNAKSPGQPIQVVYVPSHLYHMVFELFKNAMRATMEHHANRGVYPPIQVHVTLGNEDLTVKMSDRGGGVPLRKIDRLFNYMYSTAPRPRVETSRAVPLAGFGYGLPISRLYAQYFQGDLKLYSLEGYGTDAVIYIKALSTDSIERLPVYNKAAWKHYNTNHEADDWCVPSREPKDMTTFRSA",
# PDK2_HUMAN
"MRWVWALLKNASLAGAPKYIEHFSKFSPSPLSMKQFLDFGSSNACEKTSFTFLRQELPVRLANIMKEINLLPDRVLSTPSVQLVQSWYVQSLLDIMEFLDKDPEDHRTLSQFTDALVTIRNRHNDVVPTMAQGVLEYKDTYGDDPVSNQNIQYFLDRFYLSRISIRMLINQHTLIFDGSTNPAHPKHIGSIDPNCNVSEVVKDAYDMAKLLCDKYYMASPDLEIQEINAANSKQPIHMVYVPSHLYHMLFELFKNAMRATVESHESSLILPPIKVMVALGEEDLSIKMSDRGGGVPLRKIERLFSYMYSTAPTPQPGTGGTPLAGFGYGLPISRLYAKYFQGDLQLFSMEGFGTDAVIYLKALSTDSVERLPVYNKSAWRHYQTIQEAGDWCVPSTEPKNTSTYRVS",
# PDK3_HUMAN
"MRLFRWLLKQPVPKQIERYSRFSPSPLSIKQFLDFGRDNACEKTSYMFLRKELPVRLANTMREVNLLPDNLLNRPSVGLVQSWYMQSFLELLEYENKSPEDPQVLDNFLQVLIKVRNRHNDVVPTMAQGVIEYKEKFGFDPFISTNIQYFLDRFYTNRISFRMLINQHTLLFGGDTNPVHPKHIGSIDPTCNVADVVKDAYETAKMLCEQYYLVAPELEVEEFNAKAPDKPIQVVYVPSHLFHMLFELFKNSMRATVELYEDRKEGYPAVKTLVTLGKEDLSIKISDLGGGVPLRKIDRLFNYMYSTAPRPSLEPTRAAPLAGFGYGLPISRLYARYFQGDLKLYSMEGVGTDAVIYLKALSSESFERLPVFNKSAWRHYKTTPEADDWSNPSSEPRDASKYKAKQ",
# PDK4_HUMAN
"MKAARFVLRSAGSLNGAGLVPREVEHFSRYSPSPLSMKQLLDFGSENACERTSFAFLRQELPVRLANILKEIDILPTQLVNTSSVQLVKSWYIQSLMDLVEFHEKSPDDQKALSDFVDTLIKVRNRHHNVVPTMAQGIIEYKDACTVDPVTNQNLQYFLDRFYMNRISTRMLMNQHILIFSDSQTGNPSHIGSIDPNCDVVAVVQDAFECSRMLCDQYYLSSPELKLTQVNGKFPDQPIHIVYVPSHLHHMLFELFKNAMRATVEHQENQPSLTPIEVIVVLGKEDLTIKISDRGGGVPLRIIDRLFSYTYSTAPTPVMDNSRNAPLAGFGYGLPISRLYAKYFQGDLNLYSLSGYGTDAIIYLKALSSESIEKLPVFNKSAFKHYQMSSEADDWCIPSREPKNLAKEVAM",
# E2AK3_HUMAN
"MERAISPGLLVRALLLLLLLLGLAARTVAAGRARGLPAPTAEAAFGLGAAAAPTSATRVPAAGAVAAAEVTVEDAEALPAAAGEQEPRGPEPDDETELRPRGRSLVIISTLDGRIAALDPENHGKKQWDLDVGSGSLVSSSLSKPEVFGNKMIIPSLDGALFQWDQDRESMETVPFTVESLLESSYKFGDDVVLVGGKSLTTYGLSAYSGKVRYICSALGCRQWDSDEMEQEEDILLLQRTQKTVRAVGPRSGNEKWNFSVGHFELRYIPDMETRAGFIESTFKPNENTEESKIISDVEEQEAAIMDIVIKVSVADWKVMAFSKKGGHLEWEYQFCTPIASAWLLKDGKVIPISLFDDTSYTSNDDVLEDEEDIVEAARGATENSVYLGMYRGQLYLQSSVRISEKFPSSPKALESVTNENAIIPLPTIKWKPLIHSPSRTPVLVGSDEFDKCLSNDKFSHEEYSNGALSILQYPYDNGYYLPYYKRERNKRSTQITVRFLDNPHYNKNIRKKDPVLLLHWWKEIVATILFCIIATTFIVRRLFHPHPHRQRKESETQCQTENKYDSVSGEANDSSWNDIKNSGYISRYLTDFEPIQCLGRGGFGVVFEAKNKVDDCNYAIKRIRLPNRELAREKVMREVKALAKLEHPGIVRYFNAWLEAPPEKWQEKMDEIWLKDESTDWPLSSPSPMDAPSVKIRRMDPFATKEHIEIIAPSPQRSRSFSVGISCDQTSSSESQFSPLEFSGMDHEDISESVDAAYNLQDSCLTDCDVEDGTMDGNDEGHSFELCPSEASPYVRSRERTSSSIVFEDSGCDNASSKEEPKTNRLHIGNHCANKLTAFKPTSSKSSSEATLSISPPRPTTLSLDLTKNTTEKLQPSSPKVYLYIQMQLCRKENLKDWMNGRCTIEERERSVCLHIFLQIAEAVEFLHSKGLMHRDLKPSNIFFTMDDVVKVGDFGLVTAMDQDEEEQTVLTPMPAYARHTGQVGTKLYMSPEQIHGNSYSHKVDIFSLGLILFELLYPFSTQMERVRTLTDVRNLKFPPLFTQKYPCEYVMVQDMLSPSPMERPEAINIIENAVFEDLDFPGKTVLRQRSRSLSSSGTKHSRQSNNSHSPLPSN",
# CDK14_HUMAN
"MCDLIEPQPAEKIGKMKKLRRTLSESFSRIALKKDDTTFDEICVTKMSTRNCQGMDSVIKPLDTIPEDKKVRVQRTQSTFDPFEKPANQVKRVHSENNACINFKTSSTGKESPKVRRHSSPSSPTSPKFGKADSYEKLEKLGEGSYATVYKGKSKVNGKLVALKVIRLQEEEGTPFTAIREASLLKGLKHANIVLLHDIIHTKETLTLVFEYVHTDLCQYMDKHPGGLHPDNVKLFLFQLLRGLSYIHQRYILHRDLKPQNLLISDTGELKLADFGLARAKSVPSHTYSNEVVTLWYRPPDVLLGSTEYSTCLDMWGVGCIFVEMIQGVAAFPGMKDIQDQLERIFLVLGTPNEDTWPGVHSLPHFKPERFTLYSSKNLRQAWNKLSYVNHAEDLASKLLQCSPKNRLSAQAALSHEYFSDLPPRLWELTDMSSIFTVPNVRLQPEAGESMRAFGKNNSYGKSLSNSKH",
# PHKG1_HUMAN
"MTRDEALPDSHSAQDFYENYEPKEILGRGVSSVVRRCIHKPTSQEYAVKVIDVTGGGSFSPEEVRELREATLKEVDILRKVSGHPNIIQLKDTYETNTFFFLVFDLMKRGELFDYLTEKVTLSEKETRKIMRALLEVICTLHKLNIVHRDLKPENILLDDNMNIKLTDFGFSCQLEPGERLREVCGTPSYLAPEIIECSMNEDHPGYGKEVDMWSTGVIMYTLLAGSPPFWHRKQMLMLRMIMSGNYQFGSPEWDDYSDTVKDLVSRFLVVQPQNRYTAEEALAHPFFQQYLVEEVRHFSPRGKFKVIALTVLASVRIYYQYRRVKPVTREIVIRDPYALRPLRRLIDAYAFRIYGHWVKKGQQQNRAALFENTPKAVLLSLAEEDY",
# PHKG2_HUMAN
"MTLDVGPEDELPDWAAAKEFYQKYDPKDVIGRGVSSVVRRCVHRATGHEFAVKIMEVTAERLSPEQLEEVREATRRETHILRQVAGHPHIITLIDSYESSSFMFLVFDLMRKGELFDYLTEKVALSEKETRSIMRSLLEAVSFLHANNIVHRDLKPENILLDDNMQIRLSDFGFSCHLEPGEKLRELCGTPGYLAPEILKCSMDETHPGYGKEVDLWACGVILFTLLAGSPPFWHRRQILMLRMIMEGQYQFSSPEWDDRSSTVKDLISRLLQVDPEARLTAEQALQHPFFERCEGSQPWNLTPRQRFRVAVWTVLAAGRVALSTHRVRPLTKNALLRDPYALRSVRHLIDNCAFRLYGHWVKKGEQQNRAALFQHRPPGPFPIMGPEEEGDSAAITEDEAVLVLG",
# PIM1_HUMAN
"MLLSKINSLAHLRAAPCNDLHATKLAPGKEKEPLESQYQVGPLLGSGGFGSVYSGIRVSDNLPVAIKHVEKDRISDWGELPNGTRVPMEVVLLKKVSSGFSGVIRLLDWFERPDSFVLILERPEPVQDLFDFITERGALQEELARSFFWQVLEAVRHCHNCGVLHRDIKDENILIDLNRGELKLIDFGSGALLKDTVYTDFDGTRVYSPPEWIRYHRYHGRSAAVWSLGILLYDMVCGDIPFEHDEEIIRGQVFFRQRVSSECQHLIRWCLALRPSDRPTFEEIQNHPWMQDVLLPQETAEIHLHSLSPGPSK",
# PIM2_HUMAN
"MLTKPLQGPPAPPGTPTPPPGGKDREAFEAEYRLGPLLGKGGFGTVFAGHRLTDRLQVAIKVIPRNRVLGWSPLSDSVTCPLEVALLWKVGAGGGHPGVIRLLDWFETQEGFMLVLERPLPAQDLFDYITEKGPLGEGPSRCFFGQVVAAIQHCHSRGVVHRDIKDENILIDLRRGCAKLIDFGSGALLHDEPYTDFDGTRVYSPPEWISRHQYHALPATVWSLGILLYDMVCGDIPFERDQEILEAELHFPAHVSPDCCALIRRCLAPKPSSRPSLEEILLDPWMQTPAEDVPLNPSKGGPAPLAWSLLP",
# CDK10_HUMAN
"MAEPDLECEQIRLKCIRKEGFFTVPPEHRLGRCRSVKEFEKLNRIGEGTYGIVYRARDTQTDEIVALKKVRMDKEKDGIPISSLREITLLLRLRHPNIVELKEVVVGNHLESIFLVMGYCEQDLASLLENMPTPFSEAQVKCIVLQVLRGLQYLHRNFIIHRDLKVSNLLMTDKGCVKTADFGLARAYGVPVKPMTPKVVTLWYRAPELLLGTTTQTTSIDMWAVGCILAELLAHRPLLPGTSEIHQIDLIVQLLGTPSENIWPGFSKLPLVGQYSLRKQPYNNLKHKFPWLSEAGLRLLHFLFMYDPKKRATAGDCLESSYFKEKPLPCEPELMPTFPHHRNKRAAPATSEGQSKRCKP",
# CDK9_HUMAN
"MAKQYDSVECPFCDEVSKYEKLAKIGQGTFGEVFKARHRKTGQKVALKKVLMENEKEGFPITALREIKILQLLKHENVVNLIEICRTKASPYNRCKGSIYLVFDFCEHDLAGLLSNVLVKFTLSEIKRVMQMLLNGLYYIHRNKILHRDMKAANVLITRDGVLKLADFGLARAFSLAKNSQPNRYTNRVVTLWYRPPELLLGERDYGPPIDLWGAGCIMAEMWTRSPIMQGNTEQHQLALISQLCGSITPEVWPNVDNYELYEKLELVKGQKRKVKDRLKAYVRDPYALDLIDKLLVLDPAQRIDSDDALNHDFFWSDPMPSDLKGMLSTHLTSMFEYLAPPRRKGSQITQQSTNQSRNPATTNQTEFERVF",
# CD11B_HUMAN
"MGDEKDSWKVKTLDEILQEKKRRKEQEEKAEIKRLKNSDDRDSKRDSLEEGELRDHRMEITIRNSPYRREDSMEDRGEEDDSLAIKPPQQMSRKEKAHHRKDEKRKEKRRHRSHSAEGGKHARVKEKEREHERRKRHREEQDKARREWERQKRREMAREHSRRERDRLEQLERKRERERKMREQQKEQREQKERERRAEERRKEREARREVSAHHRTMREDYSDKVKASHWSRSPPRPPRERFELGDGRKPGEARPAPAQKPAQLKEEKMEERDLLSDLQDISDSERKTSSAESSSAESGSGSEEEEEEEEEEEEEGSTSEESEEEEEEEEEEEEETGSNSEEASEQSAEEVSEEEMSEDEERENENHLLVVPESRFDRDSGESEEAEEEVGEGTPQSSALTEGDYVPDSPALSPIELKQELPKYLPALQGCRSVEEFQCLNRIEEGTYGVVYRAKDKKTDEIVALKRLKMEKEKEGFPITSLREINTILKAQHPNIVTVREIVVGSNMDKIYIVMNYVEHDLKSLMETMKQPFLPGEVKTLMIQLLRGVKHLHDNWILHRDLKTSNLLLSHAGILKVGDFGLAREYGSPLKAYTPVVVTLWYRAPELLLGAKEYSTAVDMWSVGCIFGELLTQKPLFPGKSEIDQINKVFKDLGTPSEKIWPGYSELPAVKKMTFSEHPYNNLRKRFGALLSDQGFDLMNKFLTYFPGRRISAEDGLKHEYFRETPLPIDPSMFPTWPAKSEQQRVKRGTSPRPPEGGLGYSQLGDDDLKETGFHLTTTNQGASAAGPGFSLKF",
# MELK_HUMAN
"MKDYDELLKYYELHETIGTGGFAKVKLACHILTGEMVAIKIMDKNTLGSDLPRIKTEIEALKNLRHQHICQLYHVLETANKIFMVLEYCPGGELFDYIISQDRLSEEETRVVFRQIVSAVAYVHSQGYAHRDLKPENLLFDEYHKLKLIDFGLCAKPKGNKDYHLQTCCGSLAYAAPELIQGKSYLGSEADVWSMGILLYVLMCGFLPFDDDNVMALYKKIMRGKYDVPKWLSPSSILLLQQMLQVDPKKRISMKNLLNHPWIMQDYNYPVEWQSKNPFIHLDDDCVTELSVHHRNNRQTMEDLISLWQYDHLTATYLLLLAKKARGKPVRLRLSSFSCGQASATPFTDIKSNNWSLEDVTASDKNYVAGLIDYDWCEDDLSTGAATPRTSQFTKYWTESNGVESKSLTPALCRTPANKLKNKENVYTPKSAVKNEEYFMFPEPKTPVNKNQHKREILTTPNRYTTPSKARNQCLKETPIKIPVNSTGTDKLMTGVISPERRCRSVELDLNQAHMEETPKRKGAKVFGSLERGLDKVITVLTRSKRKGSARDGPRRLKLHYNVTTTRLVNPDQLLNEIMSILPKKHVDFVQKGYTLKCQTQSDFGKVTMQFELEVCQLQKPDVVGIRRQRLKGDAWVYKRLVEDILSSCKV",
# MRCKA_HUMAN
"MSGEVRLRQLEQFILDGPAQTNGQCFSVETLLDILICLYDECNNSPLRREKNILEYLEWAKPFTSKVKQMRLHREDFEILKVIGRGAFGEVAVVKLKNADKVFAMKILNKWEMLKRAETACFREERDVLVNGDNKWITTLHYAFQDDNNLYLVMDYYVGGDLLTLLSKFEDRLPEDMARFYLAEMVIAIDSVHQLHYVHRDIKPDNILMDMNGHIRLADFGSCLKLMEDGTVQSSVAVGTPDYISPEILQAMEDGKGRYGPECDWWSLGVCMYEMLYGETPFYAESLVETYGKIMNHKERFQFPAQVTDVSENAKDLIRRLICSREHRLGQNGIEDFKKHPFFSGIDWDNIRNCEAPYIPEVSSPTDTSNFDVDDDCLKNSETMPPPTHTAFSGHHLPFVGFTYTSSCVLSDRSCLRVTAGPTSLDLDVNVQRTLDNNLATEAYERRIKRLEQEKLELSRKLQESTQTVQALQYSTVDGPLTASKDLEIKNLKEEIEKLRKQVTESSHLEQQLEEANAVRQELDDAFRQIKAYEKQIKTLQQEREDLNKELVQASERLKNQSKELKDAHCQRKLAMQEFMEINERLTELHTQKQKLARHVRDKEEEVDLVMQKVESLRQELRRTERAKKELEVHTEALAAEASKDRKLREQSEHYSKQLENELEGLKQKQISYSPGVCSIEHQQEITKLKTDLEKKSIFYEEELSKREGIHANEIKNLKKELHDSEGQQLALNKEIMILKDKLEKTRRESQSEREEFESEFKQQYEREKVLLTEENKKLTSELDKLTTLYENLSIHNQQLEEEVKDLADKKESVAHWEAQITEIIQWVSDEKDARGYLQALASKMTEELEALRNSSLGTRATDMPWKMRRFAKLDMSARLELQSALDAEIRAKQAIQEELNKVKASNIITECKLKDSEKKNLELLSEIEQLIKDTEELRSEKGIEHQDSQHSFLAFLNTPTDALDQFERSPSCTPASKGRRTVDSTPLSVHTPTLRKKGCPGSTGFPPKRKTHQFFVKSFTTPTKCHQCTSLMVGLIRQGCSCEVCGFSCHITCVNKAPTTCPVPPEQTKGPLGIDPQKGIGTAYEGHVRIPKPAGVKKGWQRALAIVCDFKLFLYDIAEGKASQPSVVISQVIDMRDEEFSVSSVLASDVIHASRKDIPCIFRVTASQLSASNNKCSILMLADTENEKNKWVGVLSELHKILKKNKFRDRSVYVPKEAYDSTLPLIKTTQAAAIIDHERIALGNEEGLFVVHVTKDEIIRVGDNKKIHQIELIPNDQLVAVISGRNRHVRLFPMSALDGRETDFYKLSETKGCQTVTSGKVRHGALTCLCVAMKRQVLCYELFQSKTRHRKFKEIQVPYNVQWMAIFSEQLCVGFQSGFLRYPLNGEGNPYSMLHSNDHTLSFIAHQPMDAICAVEISSKEYLLCFNSIGIYTDCQGRRSRQQELMWPANPSSCCYNAPYLSVYSENAVDIFDVNSMEWIQTLPLKKVRPLNNEGSLNLLGLETIRLIYFKNKMAEGDELVVPETSDNSRKQMVRNINNKRRYSFRVPEEERMQQRREMLRDPEMRNKLISNPTNFNHIAHMGPGDGIQILKDLPMNPRPQESRTVFSGSVSIPSITKSRPEPGRSMSASSGLSARSSAQNGSALKREFSGGSYSAKRQPMPSPSEGSLSSGGMDQGSDAPARDFDGEDSDSPRHSTASNSSNLSSPPSPASPRKTKSLSLESTDRGSWDP",
# KAPCA_HUMAN
"MGNAAAAKKGSEQESVKEFLAKAKEDFLKKWESPAQNTAHLDQFERIKTLGTGSFGRVMLVKHKETGNHYAMKILDKQKVVKLKQIEHTLNEKRILQAVNFPFLVKLEFSFKDNSNLYMVMEYVPGGEMFSHLRRIGRFSEPHARFYAAQIVLTFEYLHSLDLIYRDLKPENLLIDQQGYIQVTDFGFAKRVKGRTWTLCGTPEYLAPEIILSKGYNKAVDWWALGVLIYEMAAGYPPFFADQPIQIYEKIVSGKVRFPSHFSSDLKDLLRNLLQVDLTKRFGNLKNGVNDIKNHKWFATTDWIAIYQRKVEAPFIPKFKGPGDTSNFDDYEEEEIRVSINEKCGKEFSEF",
# KAPCB_HUMAN
"MGNAATAKKGSEVESVKEFLAKAKEDFLKKWENPTQNNAGLEDFERKKTLGTGSFGRVMLVKHKATEQYYAMKILDKQKVVKLKQIEHTLNEKRILQAVNFPFLVRLEYAFKDNSNLYMVMEYVPGGEMFSHLRRIGRFSEPHARFYAAQIVLTFEYLHSLDLIYRDLKPENLLIDHQGYIQVTDFGFAKRVKGRTWTLCGTPEYLAPEIILSKGYNKAVDWWALGVLIYEMAAGYPPFFADQPIQIYEKIVSGKVRFPSHFSSDLKDLLRNLLQVDLTKRFGNLKNGVSDIKTHKWFATTDWIAIYQRKVEAPFIPKFRGSGDTSNFDDYEEEDIRVSITEKCAKEFGEF",
# KAPCG_HUMAN
"MGNAPAKKDTEQEESVNEFLAKARGDFLYRWGNPAQNTASSDQFERLRTLGMGSFGRVMLVRHQETGGHYAMKILNKQKVVKMKQVEHILNEKRILQAIDFPFLVKLQFSFKDNSYLYLVMEYVPGGEMFSRLQRVGRFSEPHACFYAAQVVLAVQYLHSLDLIHRDLKPENLLIDQQGYLQVTDFGFAKRVKGRTWTLCGTPEYLAPEIILSKGYNKAVDWWALGVLIYEMAVGFPPFYADQPIQIYEKIVSGRVRFPSKLSSDLKHLLRSLLQVDLTKRFGNLRNGVGDIKNHKWFATTSWIAIYEKKVEAPFIPKYTGPGDASNFDDYEEEELRISINEKCAKEFSEF",
# KPCA_HUMAN
"MADVFPGNDSTASQDVANRFARKGALRQKNVHEVKDHKFIARFFKQPTFCSHCTDFIWGFGKQGFQCQVCCFVVHKRCHEFVTFSCPGADKGPDTDDPRSKHKFKIHTYGSPTFCDHCGSLLYGLIHQGMKCDTCDMNVHKQCVINVPSLCGMDHTEKRGRIYLKAEVADEKLHVTVRDAKNLIPMDPNGLSDPYVKLKLIPDPKNESKQKTKTIRSTLNPQWNESFTFKLKPSDKDRRLSVEIWDWDRTTRNDFMGSLSFGVSELMKMPASGWYKLLNQEEGEYYNVPIPEGDEEGNMELRQKFEKAKLGPAGNKVISPSEDRKQPSNNLDRVKLTDFNFLMVLGKGSFGKVMLADRKGTEELYAIKILKKDVVIQDDDVECTMVEKRVLALLDKPPFLTQLHSCFQTVDRLYFVMEYVNGGDLMYHIQQVGKFKEPQAVFYAAEISIGLFFLHKRGIIYRDLKLDNVMLDSEGHIKIADFGMCKEHMMDGVTTRTFCGTPDYIAPEIIAYQPYGKSVDWWAYGVLLYEMLAGQPPFDGEDEDELFQSIMEHNVSYPKSLSKEAVSVCKGLMTKHPAKRLGCGPEGERDVREHAFFRRIDWEKLENREIQPPFKPKVCGKGAENFDKFFTRGQPVLTPPDQLVIANIDQSDFEGFSYVNPQFVHPILQSAV",
# KPCB_HUMAN
"MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQPTFCSHCTDFIWGFGKQGFQCQVCCFVVHKRCHEFVTFSCPGADKGPASDDPRSKHKFKIHTYSSPTFCDHCGSLLYGLIHQGMKCDTCMMNVHKRCVMNVPSLCGTDHTERRGRIYIQAHIDRDVLIVLVRDAKNLVPMDPNGLSDPYVKLKLIPDPKSESKQKTKTIKCSLNPEWNETFRFQLKESDKDRRLSVEIWDWDLTSRNDFMGSLSFGISELQKASVDGWFKLLSQEEGEYFNVPVPPEGSEANEELRQKFERAKISQGTKVPEEKTTNTVSKFDNNGNRDRMKLTDFNFLMVLGKGSFGKVMLSERKGTDELYAVKILKKDVVIQDDDVECTMVEKRVLALPGKPPFLTQLHSCFQTMDRLYFVMEYVNGGDLMYHIQQVGRFKEPHAVFYAAEIAIGLFFLQSKGIIYRDLKLDNVMLDSEGHIKIADFGMCKENIWDGVTTKTFCGTPDYIAPEIIAYQPYGKSVDWWAFGVLLYEMLAGQAPFEGEDEDELFQSIMEHNVAYPKSMSKEAVAICKGLMTKHPGKRLGCGPEGERDIKEHAFFRYIDWEKLERKEIQPPYKPKARDKRDTSNFDKEFTRQPVELTPTDKLFIMNLDQNEFAGFSYTNPEFVINV",
# KPCD_HUMAN
"MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQKKPTMYPEWKSTFDAHIYEGRVIQIVLMRAAEEPVSEVTVGVSVLAERCKKNNGKAEFWLDLQPQAKVLMSVQYFLEDVDCKQSMRSEDEAKFPTMNRRGAIKQAKIHYIKNHEFIATFFGQPTFCSVCKDFVWGLNKQGYKCRQCNAAIHKKCIDKIIGRCTGTAANSRDTIFQKERFNIDMPHRFKVHNYMSPTFCDHCGSLLWGLVKQGLKCEDCGMNVHHKCREKVANLCGINQKLLAEALNQVTQRASRRSDSASSEPVGIYQGFEKKTGVAGEDMQDNSGTYGKIWEGSSKCNINNFIFHKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVEKRVLTLAAENPFLTHLICTFQTKDHLFFVMEFLNGGDLMYHIQDKGRFELYRATFYAAEIMCGLQFLHSKGIIYRDLKLDNVLLDRDGHIKIADFGMCKENIFGESRASTFCGTPDYIAPEILQGLKYTFSVDWWSFGVLLYEMLIGQSPFHGDDEDELFESIRVDTPHYPRWITKESKDILEKLFEREPTKRLGVTGNIKIHPFFKTINWTLLEKRRLEPPFRPKVKSPRDYSNFDQEFLNEKARLSYSDKNLIDSMDQSAFAGFSFVNPKFEHLLED",
# KPCE_HUMAN
"MVVFNGLLKIKICEAVSLKPTAWSLRHAVGPRPQTFLLDPYIALNVDDSRIGQTATKQKTNSPAWHDEFVTDVCNGRKIELAVFHDAPIGYDDFVANCTIQFEELLQNGSRHFEDWIDLEPEGRVYVIIDLSGSSGEAPKDNEERVFRERMRPRKRQGAVRRRVHQVNGHKFMATYLRQPTYCSHCRDFIWGVIGKQGYQCQVCTCVVHKRCHELIITKCAGLKKQETPDQVGSQRFSVNMPHKFGIHNYKVPTFCDHCGSLLWGLLRQGLQCKVCKMNVHRRCETNVAPNCGVDARGIAKVLADLGVTPDKITNSGQRRKKLIAGAESPQPASGSSPSEEDRSKSAPTSPCDQEIKELENNIRKALSFDNRGEEHRAASSPDGQLMSPGENGEVRQGQAKRLGLDEFNFIKVLGKGSFGKVMLAELKGKDEVYAVKVLKKDVILQDDDVDCTMTEKRILALARKHPYLTQLYCCFQTKDRLFFVMEYVNGGDLMFQIQRSRKFDEPRSRFYAAEVTSALMFLHQHGVIYRDLKLDNILLDAEGHCKLADFGMCKEGILNGVTTTTFCGTPDYIAPEILQELEYGPSVDWWALGVLMYEMMAGQPPFEADNEDDLFESILHDDVLYPVWLSKEAVSILKAFMTKNPHKRLGCVASQNGEDAIKQHPFFKEIDWVLLEQKKIKPPFKPRIKTKRDVNNFDQDFTREEPVLTLVDEAIVKQINQEEFKGFSYFGEDLMP",
# KPCG_HUMAN
"MAGLGPGVGDSEGGPRPLFCRKGALRQKVVHEVKSHKFTARFFKQPTFCSHCTDFIWGIGKQGLQCQVCSFVVHRRCHEFVTFECPGAGKGPQTDDPRNKHKFRLHSYSSPTFCDHCGSLLYGLVHQGMKCSCCEMNVHRRCVRSVPSLCGVDHTERRGRLQLEIRAPTADEIHVTVGEARNLIPMDPNGLSDPYVKLKLIPDPRNLTKQKTRTVKATLNPVWNETFVFNLKPGDVERRLSVEVWDWDRTSRNDFMGAMSFGVSELLKAPVDGWYKLLNQEEGEYYNVPVADADNCSLLQKFEACNYPLELYERVRMGPSSSPIPSPSPSPTDPKRCFFGASPGRLHISDFSFLMVLGKGSFGKVMLAERRGSDELYAIKILKKDVIVQDDDVDCTLVEKRVLALGGRGPGGRPHFLTQLHSTFQTPDRLYFVMEYVTGGDLMYHIQQLGKFKEPHAAFYAAEIAIGLFFLHNQGIIYRDLKLDNVMLDAEGHIKITDFGMCKENVFPGTTTRTFCGTPDYIAPEIIAYQPYGKSVDWWSFGVLLYEMLAGQPPFDGEDEEELFQAIMEQTVTYPKSLSREAVAICKGFLTKHPGKRLGSGPDGEPTIRAHGFFRWIDWERLERLEIPPPFRPRPCGRSGENFDKFFTRAAPALTPPDRLVLASIDQADFQGFTYVNPDFVHPDARSPTSPVPVPVM",
# KPCI_HUMAN
"MPTQRDSSTMSHTVAGGGSGDHSHQVRVKAYYRGDIMITHFEPSISFEGLCNEVRDMCSFDNEQLFTMKWIDEEGDPCTVSSQLELEEAFRLYELNKDSELLIHVFPCVPERPGMPCPGEDKSIYRRGARRWRKLYCANGHTFQAKRFNRRAHCAICTDRIWGLGRQGYKCINCKLLVHKKCHKLVTIECGRHSLPQEPVMPMDQSSMHSDHAQTVIPYNPSSHESLDQVGEEKEAMNTRESGKASSSLGLQDFDLLRVIGRGSYAKVLLVRLKKTDRIYAMKVVKKELVNDDEDIDWVQTEKHVFEQASNHPFLVGLHSCFQTESRLFFVIEYVNGGDLMFHMQRQRKLPEEHARFYSAEISLALNYLHERGIIYRDLKLDNVLLDSEGHIKLTDYGMCKEGLRPGDTTSTFCGTPNYIAPEILRGEDYGFSVDWWALGVLMFEMMAGRSPFDIVGSSDNPDQNTEDYLFQVILEKQIRIPRSLSVKAASVLKSFLNKDPKERLGCHPQTGFADIQGHPFFRNVDWDMMEQKQVVPPFKPNISGEFGLDNFDSQFTNEPVQLTPDDDDIVRKIDQSEFEGFEYINPLLMSAEECV",
# KPCD1_HUMAN
"MSAPPVLRPPSPLLPVAAAAAAAAAALVPGSGPGPAPFLAPVAAPVGGISFHLQIGLSREPVLLLQDSSGDYSLAHVREMACSIVDQKFPECGFYGMYDKILLFRHDPTSENILQLVKAASDIQEGDLIEVVLSASATFEDFQIRPHALFVHSYRAPAFCDHCGEMLWGLVRQGLKCEGCGLNYHKRCAFKIPNNCSGVRRRRLSNVSLTGVSTIRTSSAELSTSAPDEPLLQKSPSESFIGREKRSNSQSYIGRPIHLDKILMSKVKVPHTFVIHSYTRPTVCQYCKKLLKGLFRQGLQCKDCRFNCHKRCAPKVPNNCLGEVTINGDLLSPGAESDVVMEEGSDDNDSERNSGLMDDMEEAMVQDAEMAMAECQNDSGEMQDPDPDHEDANRTISPSTSNNIPLMRVVQSVKHTKRKSSTVMKEGWMVHYTSKDTLRKRHYWRLDSKCITLFQNDTGSRYYKEIPLSEILSLEPVKTSALIPNGANPHCFEITTANVVYYVGENVVNPSSPSPNNSVLTSGVGADVARMWEIAIQHALMPVIPKGSSVGTGTNLHRDISVSISVSNCQIQENVDISTVYQIFPDEVLGSGQFGIVYGGKHRKTGRDVAIKIIDKLRFPTKQESQLRNEVAILQNLHHPGVVNLECMFETPERVFVVMEKLHGDMLEMILSSEKGRLPEHITKFLITQILVALRHLHFKNIVHCDLKPENVLLASADPFPQVKLCDFGFARIIGEKSFRRSVVGTPAYLAPEVLRNKGYNRSLDMWSVGVIIYVSLSGTFPFNEDEDIHDQIQNAAFMYPPNPWKEISHEAIDLINNLLQVKMRKRYSVDKTLSHPWLQDYQTWLDLRELECKIGERYITHESDDLRWEKYAGEQGLQYPTHLINPSASHSDTPETEETEMKALGERVSIL",
# KPCT_HUMAN
"MSPFLRIGLSNFDCGSCQSCQGEAVNPYCAVLVKEYVESENGQMYIQKKPTMYPPWDSTFDAHINKGRVMQIIVKGKNVDLISETTVELYSLAERCRKNNGKTEIWLELKPQGRMLMNARYFLEMSDTKDMNEFETEGFFALHQRRGAIKQAKVHHVKCHEFTATFFPQPTFCSVCHEFVWGLNKQGYQCRQCNAAIHKKCIDKVIAKCTGSAINSRETMFHKERFKIDMPHRFKVYNYKSPTFCEHCGTLLWGLARQGLKCDACGMNVHHRCQTKVANLCGINQKLMAEALAMIESTQQARCLRDTEQIFREGPVEIGLPCSIKNEARPPCLPTPGKREPQGISWESPLDEVDKMCHLPEPELNKERPSLQIKLKIEDFILHKMLGKGSFGKVFLAEFKKTNQFFAIKALKKDVVLMDDDVECTMVEKRVLSLAWEHPFLTHMFCTFQTKENLFFVMEYLNGGDLMYHIQSCHKFDLSRATFYAAEIILGLQFLHSKGIVYRDLKLDNILLDKDGHIKIADFGMCKENMLGDAKTNTFCGTPDYIAPEILLGQKYNHSVDWWSFGVLLYEMLIGQSPFHGQDEEELFHSIRMDNPFYPRWLEKEAKDLLVKLFVREPEKRLGVRGDIRQHPLFREINWEELERKEIDPPFRPKVKSPFDCSNFDKEFLNEKPRLSFADRALINSMDQNMFRNFSFMNPGMERLIS",
# KPCZ_HUMAN
"MPSRTGPKMEGSGGRVRLKAHYGGDIFITSVDAATTFEELCEEVRDMCRLHQQHPLTLKWVDSEGDPCTVSSQMELEEAFRLARQCRDEGLIIHVFPSTPEQPGLPCPGEDKSIYRRGARRWRKLYRANGHLFQAKRFNRRAYCGQCSERIWGLARQGYRCINCKLLVHKRCHGLVPLTCRKHMDSVMPSQEPPVDDKNEDADLPSEETDGIAYISSSRKHDSIKDDSEDLKPVIDGMDGIKISQGLGLQDFDLIRVIGRGSYAKVLLVRLKKNDQIYAMKVVKKELVHDDEDIDWVQTEKHVFEQASSNPFLVGLHSCFQTTSRLFLVIEYVNGGDLMFHMQRQRKLPEEHARFYAAEICIALNFLHERGIIYRDLKLDNVLLDADGHIKLTDYGMCKEGLGPGDTTSTFCGTPNYIAPEILRGEEYGFSVDWWALGVLMFEMMAGRSPFDIITDNPDMNTEDYLFQVILEKPIRIPRFLSVKASHVLKGFLNKDPKERLGCRPQTGFSDIKSHAFFRSIDWDLLEKKQALPPFQPQITDDYGLDNFDTQFTSEPVQLTPDDEDAIKRIDQSEFEGFEYINPLLLSTEESV",
# PRKX_HUMAN
"MEAPGLAQAAAAESDSRKVAEETPDGAPALCPSPEALSPEPPVYSLQDFDTLATVGTGTFGRVHLVKEKTAKHFFALKVMSIPDVIRLKQEQHVHNEKSVLKEVSHPFLIRLFWTWHDERFLYMLMEYVPGGELFSYLRNRGRFSSTTGLFYSAEIICAIEYLHSKEIVYRDLKPENILLDRDGHIKLTDFGFAKKLVDRTWTLCGTPEYLAPEVIQSKGHGRAVDWWALGILIFEMLSGFPPFFDDNPFGIYQKILAGKIDFPRHLDFHVKDLIKKLLVVDRTRRLGNMKNGANDVKHHRWFRSVDWEAVPQRKLKPPIVPKIAGDGDTSNFETYPENDWDTAAPVPQKDLEIFKNF",
# HIPK3_HUMAN
"MASQVLVYPPYVYQTQSSAFCSVKKLKVEPSSCVFQERNYPRTYVNGRNFGNSHPPTKGSAFQTKIPFNRPRGHNFSLQTSAVVLKNTAGATKVIAAQAQQAHVQAPQIGAWRNRLHFLEGPQRCGLKRKSEELDNHSSAMQIVDELSILPAMLQTNMGNPVTVVTATTGSKQNCTTGEGDYQLVQHEVLCSMKNTYEVLDFLGRGTFGQVVKCWKRGTNEIVAIKILKNHPSYARQGQIEVSILARLSTENADEYNFVRAYECFQHRNHTCLVFEMLEQNLYDFLKQNKFSPLPLKVIRPILQQVATALKKLKSLGLIHADLKPENIMLVDPVRQPYRVKVIDFGSASHVSKTVCSTYLQSRYYRAPEIILGLPFCEAIDMWSLGCVIAELFLGWPLYPGALEYDQIRYISQTQGLPGEQLLNVGTKSTRFFCKETDMSHSGWRLKTLEEHEAETGMKSKEARKYIFNSLDDVAHVNTVMDLEGSDLLAEKADRREFVSLLKKMLLIDADLRITPAETLNHPFVNMKHLLDFPHSNHVKSCFHIMDICKSHLNSCDTNNHNKTSLLRPVASSSTATLTANFTKIGTLRSQALTTSAHSVVHHGIPLQAGTAQFGCGDAFQQTLIICPPAIQGIPATHGKPTSYSIRVDNTVPLVTQAPAVQPLQIRPGVLSQTWSGRTQQMLVPAWQQVTPLAPATTTLTSESVAGSHRLGDWGKMISCSNHYNSVMPQPLLTNQITLSAPQPVSVGIAHVVWPQPATTKKNKQCQNRGILVKLMEWEPGREEINAFSWSNSLQNTNIPHSAFISPKIINGKDVEEVSCIETQDNQNSEGEARNCCETSIRQDSDSSVSDKQRQTIIIADSPSPAVSVITISSDTDEEETSQRHSLRECKGSLDCEACQSTLNIDRMCSLSSPDSTLSTSSSGQSSPSPCKRPNSMSDEEQESSCDTVDGSPTSDSSGHDSPFAESTFVEDTHENTELVSSADTETKPAVCSVVVPPVELENGLNADEHMANTDSICQPLIKGRSAPGRLNQPSAVGTRQQKLTSAFQQQHLNFSQVQHFGSGHQEWNGNFGHRRQQAYIPTSVTSNPFTLSHGSPNHTAVHAHLAGNTHLGGQPTLLPYPSSATLSSAAPVAHLLASPCTSRPMLQHPTYNISHPSGIVHQVPVGLNPRLLPSPTIHQTQYKPIFPPHSYIAASPAYTGFPLSPTKLSQYPYM",
# PLK1_HUMAN
"MSAAVTAGKLARAPADPGKAGVPGVAAPGAPAAAPPAKEIPEVLVDPRSRRRYVRGRFLGKGGFAKCFEISDADTKEVFAGKIVPKSLLLKPHQREKMSMEISIHRSLAHQHVVGFHGFFEDNDFVFVVLELCRRRSLLELHKRRKALTEPEARYYLRQIVLGCQYLHRNRVIHRDLKLGNLFLNEDLEVKIGDFGLATKVEYDGERKKTLCGTPNYIAPEVLSKKGHSFEVDVWSIGCIMYTLLVGKPPFETSCLKETYLRIKKNEYSIPKHINPVAASLIQKMLQTDPTARPTINELLNDEFFTSGYIPARLPITCLTIPPRFSIAPSSLDPSNRKPLTVLNKGLENPLPERPREKEEPVVRETGEVVDCHLSDMLQQLHSVNASKPSERGLVRQEEAEDPACIPIFWVSKWVDYSDKYGLGYQLCDNSVGVLFNDSTRLILYNDGDSLQYIERDGTESYLTVSSHPNSLMKKITLLKYFRNYMSEHLLKAGANITPREGDELARLPYLRTWFRTRSAIILHLSNGSVQINFFQDHTKLILCPLMAAVTYIDEKRDFRTYRLSLLEEYGCCKELASRLRYARTMVDKLLSSRSASNRLKAS",
# PLK3_HUMAN
"MEPAAGFLSPRPFQRAAAAPAPPAGPGPPPSALRGPELEMLAGLPTSDPGRLITDPRSGRTYLKGRLLGKGGFARCYEATDTETGSAYAVKVIPQSRVAKPHQREKILNEIELHRDLQHRHIVRFSHHFEDADNIYIFLELCSRKSLAHIWKARHTLLEPEVRYYLRQILSGLKYLHQRGILHRDLKLGNFFITENMELKVGDFGLAARLEPPEQRKKTICGTPNYVAPEVLLRQGHGPEADVWSLGCVMYTLLCGSPPFETADLKETYRCIKQVHYTLPASLSLPARQLLAAILRASPRDRPSIDQILRHDFFTKGYTPDRLPISSCVTVPDLTPPNPARSLFAKVTKSLFGRKKKSKNHAQERDEVSGLVSGLMRTSVGHQDARPEAPAASGPAPVSLVETAPEDSSPRGTLASSGDGFEEGLTVATVVESALCALRNCIAFMPPAEQNPAPLAQPEPLVWVSKWVDYSNKFGFGYQLSSRRVAVLFNDGTHMALSANRKTVHYNPTSTKHFSFSVGAVPRALQPQLGILRYFASYMEQHLMKGGDLPSVEEVEVPAPPLLLQWVKTDQALLMLFSDGTVQVNFYGDHTKLILSGWEPLLVTFVARNRSACTYLASHLRQLGCSPDLRQRLRYALRLLRDRSPA",
# PKN1_HUMAN
"MASDAVQSEPRSWSLLEQLGLAGADLAAPGVQQQLELERERLRREIRKELKLKEGAENLRRATTDLGRSLGPVELLLRGSSRRLDLLHQQLQELHAHVVLPDPAATHDGPQSPGAGGPTCSATNLSRVAGLEKQLAIELKVKQGAENMIQTYSNGSTKDRKLLLTAQQMLQDSKTKIDIIRMQLRRALQAGQLENQAAPDDTQGSPDLGAVELRIEELRHHFRVEHAVAEGAKNVLRLLSAAKAPDRKAVSEAQEKLTESNQKLGLLREALERRLGELPADHPKGRLLREELAAASSAAFSTRLAGPFPATHYSTLCKPAPLTGTLEVRVVGCRDLPETIPWNPTPSMGGPGTPDSRPPFLSRPARGLYSRSGSLSGRSSLKAEAENTSEVSTVLKLDNTVVGQTSWKPCGPNAWDQSFTLELERARELELAVFWRDQRGLCALKFLKLEDFLDNERHEVQLDMEPQGCLVAEVTFRNPVIERIPRLRRQKKIFSKQQGKAFQRARQMNIDVATWVRLLRRLIPNATGTGTFSPGASPGSEARTTGDISVEKLNLGTDSDSSPQKSSRDPPSSPSSLSSPIQESTAPELPSETQETPGPALCSPLRKSPLTLEDFKFLAVLGRGHFGKVLLSEFRPSGELFAIKALKKGDIVARDEVESLMCEKRILAAVTSAGHPFLVNLFGCFQTPEHVCFVMEYSAGGDLMLHIHSDVFSEPRAIFYSACVVLGLQFLHEHKIVYRDLKLDNLLLDTEGYVKIADFGLCKEGMGYGDRTSTFCGTPEFLAPEVLTDTSYTRAVDWWGLGVLLYEMLVGESPFPGDDEEEVFDSIVNDEVRYPRFLSAEAIGIMRRLLRRNPERRLGSSERDAEDVKKQPFFRTLGWEALLARRLPPPFVPTLSGRTDVSNFDEEFTGEAPTLSPPRDARPLTAAEQAAFLDFDFVAGGC",
# PKN2_HUMAN
"MASNPERGEILLTELQGDSRSLPFSENVSAVQKLDFSDTMVQQKLDDIKDRIKREIRKELKIKEGAENLRKVTTDKKSLAYVDNILKKSNKKLEELHHKLQELNAHIVVSDPEDITDCPRTPDTPNNDPRCSTSNNRLKALQKQLDIELKVKQGAENMIQMYSNGSSKDRKLHGTAQQLLQDSKTKIEVIRMQILQAVQTNELAFDNAKPVISPLELRMEELRHHFRIEFAVAEGAKNVMKLLGSGKVTDRKALSEAQARFNESSQKLDLLKYSLEQRLNEVPKNHPKSRIIIEELSLVAASPTLSPRQSMISTQNQYSTLSKPAALTGTLEVRLMGCQDILENVPGRSKATSVALPGWSPSETRSSFMSRTSKSKSGSSRNLLKTDDLSNDVCAVLKLDNTVVGQTSWKPISNQSWDQKFTLELDRSRELEISVYWRDWRSLCAVKFLRLEDFLDNQRHGMCLYLEPQGTLFAEVTFFNPVIERRPKLQRQKKIFSKQQGKTFLRAPQMNINIATWGRLVRRAIPTVNHSGTFSPQAPVPTTVPVVDVRIPQLAPPASDSTVTKLDFDLEPEPPPAPPRASSLGEIDESSELRVLDIPGQDSETVFDIQNDRNSILPKSQSEYKPDTPQSGLEYSGIQELEDRRSQQRFQFNLQDFRCCAVLGRGHFGKVLLAEYKNTNEMFAIKALKKGDIVARDEVDSLMCEKRIFETVNSVRHPFLVNLFACFQTKEHVCFVMEYAAGGDLMMHIHTDVFSEPRAVFYAACVVLGLQYLHEHKIVYRDLKLDNLLLDTEGFVKIADFGLCKEGMGYGDRTSTFCGTPEFLAPEVLTETSYTRAVDWWGLGVLIYEMLVGESPFPGDDEEEVFDSIVNDEVRYPRFLSTEAISIMRRLLRRNPERRLGASEKDAEDVKKHPFFRLIDWSALMDKKVKPPFIPTIRGREDVSNFDDEFTSEAPILTPPREPRILSEEEQEMFRDFDYIADWC",
# PRKY_HUMAN
"MEAPGPAQAAAAESNSREVTEDAADWAPALCPSPEARSPEAPAYRLQDCDALVTMGTGTFGRVHLVKEKTAKHFFALKVMSIPDVIRRKQEQHVHNEKSVLKEVSHPFLIRLFWTWHEERFLYMLMEYVPGGELFSYLRNRGHFSSTTGLFYSAEIICAIEYLHSKEIVYRDLKPENILLDRDGHIKLTDFGFAKKLVDRTWTLCGTPEYLAPEVIQSKGHGRAVDWWALGILIFEMLSGFPPFFDDNPFGIYQKILAGKLYFPRHLDFHVKTGRMM",
# PRP4K_HUMAN
"MAAAETQSLREQPEMEDANSEKSINEENGEVSEDQSQNKHSRHKKKKHKHRSKHKKHKHSSEEDKDKKHKHKHKHKKHKRKEIIDASDKEGMSPAKRTKLDDLALLEDLEKQRALIKAELDNELMEGKVQSGMGLILQGYESGSEEEGEIHEKARNGNRSSTRSSSTKGKLELVDNKITTKKRSKSRSKERTRHRSDKKKSKGGIEIVKEKTTRSKSKERKKSKSPSKRSKSQDQARKSKSPTLRRRSQEKIGKARSPTDDKVKIEDKSKSKDRKKSPIINESRSRDRGKKSRSPVDLRGKSKDRRSRSKERKSKRSETDKEKKPIKSPSKDASSGKENRSPSRRPGRSPKRRSLSPKPRDKSRRSRSPLLNDRRSKQSKSPSRTLSPGRRAKSRSLERKRREPERRRLSSPRTRPRDDILSRRERSKDASPINRWSPTRRRSRSPIRRRSRSPLRRSRSPRRRSRSPRRRDRGRRSRSRLRRRSRSRGGRRRRSRSKVKEDKFKGSLSEGMKVEQESSSDDNLEDFDVEEEDEEALIEQRRIQRQAIVQKYKYLAEDSNMSVPSEPSSPQSSTRTRSPSPDDILERVAADVKEYERENVDTFEASVKAKHNLMTVEQNNGSSQKKLLAPDMFTESDDMFAAYFDSARLRAAGIGKDFKENPNLRDNWTDAEGYYRVNIGEVLDKRYNVYGYTGQGVFSNVVRARDNARANQEVAVKIIRNNELMQKTGLKELEFLKKLNDADPDDKFHCLRLFRHFYHKQHLCLVFEPLSMNLREVLKKYGKDVGLHIKAVRSYSQQLFLALKLLKRCNILHADIKPDNILVNESKTILKLCDFGSASHVADNDITPYLVSRFYRAPEIIIGKSYDYGIDMWSVGCTLYELYTGKILFPGKTNNHMLKLAMDLKGKMPNKMIRKGVFKDQHFDQNLNFMYIEVDKVTEREKVTVMSTINPTKDLLADLIGCQRLPEDQRKKVHQLKDLLDQILMLDPAKRISINQALQHAFIQEKI",
# KPSH1_HUMAN
"MGCGTSKVLPEPPKDVQLDLVKKVEPFSGTKSDVYKHFITEVDSVGPVKAGFPAASQYAHPCPGPPTAGHTEPPSEPPRRARVAKYRAKFDPRVTAKYDIKALIGRGSFSRVVRVEHRATRQPYAIKMIETKYREGREVCESELRVLRRVRHANIIQLVEVFETQERVYMVMELATGGELFDRIIAKGSFTERDATRVLQMVLDGVRYLHALGITHRDLKPENLLYYHPGTDSKIIITDFGLASARKKGDDCLMKTTCGTPEYIAPEVLVRKPYTNSVDMWALGVIAYILLSGTMPFEDDNRTRLYRQILRGKYSYSGEPWPSVSNLAKDFIDRLLTVDPGARMTALQALRHPWVVSMAASSSMKNLHRSISQNLLKRASSRCQSTKSAQSTRSSRSTRSNKSRRVRERELRELNLRYQQQYNG",
# RAF1_HUMAN
"MEHIQGAWKTISNGFGFKDAVFDGSSCISPTIVQQFGYQRRASDDGKLTDPSKTSNTIRVFLPNKQRTVVNVRNGMSLHDCLMKALKVRGLQPECCAVFRLLHEHKGKKARLDWNTDAASLIGEELQVDFLDHVPLTTHNFARKTFLKLAFCDICQKFLLNGFRCQTCGYKFHEHCSTKVPTMCVDWSNIRQLLLFPNSTIGDSGVPALPSLTMRRMRESVSRMPVSSQHRYSTPHAFTFNTSSPSSEGSLSQRQRSTSTPNVHMVSTTLPVDSRMIEDAIRSHSESASPSALSSSPNNLSPTGWSQPKTPVPAQRERAPVSGTQEKNKIRPRGQRDSSYYWEIEASEVMLSTRIGSGSFGTVYKGKWHGDVAVKILKVVDPTPEQFQAFRNEVAVLRKTRHVNILLFMGYMTKDNLAIVTQWCEGSSLYKHLHVQETKFQMFQLIDIARQTAQGMDYLHAKNIIHRDMKSNNIFLHEGLTVKIGDFGLATVKSRWSGSQQVEQPTGSVLWMAPEVIRMQDNNPFSFQSDVYSYGIVLYELMTGELPYSHINNRDQIIFMVGRGYASPDLSKLYKNCPKAMKRLVADCVKKVKEERPLFPQILSSIELLQHSLPKINRSASEPSLHRAAHTEDINACTLTTSPRLPVF",
# RET_HUMAN
"MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAAGTPLLYVHALRDAPEEVPSFRLGQHLYGTYRTRLHENNWICIQEDTGLLYLNRSLDHSSWEKLSVRNRGFPLLTVYLKVFLSPTSLREGECQWPGCARVYFSFFNTSFPACSSLKPRELCFPETRPSFRIRENRPPGTFHQFRLLPVQFLCPNISVAYRLLEGEGLPFRCAPDSLEVSTRWALDREQREKYELVAVCTVHAGAREEVVMVPFPVTVYDEDDSAPTFPAGVDTASAVVEFKRKEDTVVATLRVFDADVVPASGELVRRYTSTLLPGDTWAQQTFRVEHWPNETSVQANGSFVRATVHDYRLVLNRNLSISENRTMQLAVLVNDSDFQGPGAGVLLLHFNVSVLPVSLHLPSTYSLSVSRRARRFAQIGKVCVENCQAFSGINVQYKLHSSGANCSTLGVVTSAEDTSGILFVNDTKALRRPKCAELHYMVVATDQQTSRQAQAQLLVTVEGSYVAEEAGCPLSCAVSKRRLECEECGGLGSPTGRCEWRQGDGKGITRNFSTCSPSTKTCPDGHCDVVETQDINICPQDCLRGSIVGGHEPGEPRGIKAGYGTCNCFPEEEKCFCEPEDIQDPLCDELCRTVIAAAVLFSFIVSVLLSAFCIHCYHKFAHKPPISSAEMTFRRPAQAFPVSYSSSGARRPSLDSMENQVSVDAFKILEDPKWEFPRKNLVLGKTLGEGEFGKVVKATAFHLKGRAGYTTVAVKMLKENASPSELRDLLSEFNVLKQVNHPHVIKLYGACSQDGPLLLIVEYAKYGSLRGFLRESRKVGPGYLGSGGSRNSSSLDHPDERALTMGDLISFAWQISQGMQYLAEMKLVHRDLAARNILVAEGRKMKISDFGLSRDVYEEDSYVKRSQGRIPVKWMAIESLFDHIYTTQSDVWSFGVLLWEIVTLGGNPYPGIPPERLFNLLKTGHRMERPDNCSEEMYRLMLQCWKQEPDKRPVFADISKDLEKMMVKRRDYLDLAASTPSDSLIYDDGLSEEETPLVDCNNAPLPRALPSTWIENKLYGMSDPNWPGESPVPLTRADGTNTGFPRYPNDSVYANWMLSPSAAKLMDTFDS",
# GRK1_HUMAN
"MDFGSLETVVANSAFIAARGSFDGSSSQPSRDKKYLAKLKLPPLSKCESLRDSLSLEFESVCLEQPIGKKLFQQFLQSAEKHLPALELWKDIEDYDTADNDLQPQKAQTILAQYLDPQAKLFCSFLDEGIVAKFKEGPVEIQDGLFQPLLQATLAHLGQAPFQEYLGSLYFLRFLQWKWLEAQPMGEDWFLDFRVLGKGGFGEVSACQMKATGKLYACKKLNKKRLKKRKGYQGAMVEKKILMKVHSRFIVSLAYAFETKADLCLVMTIMNGGDIRYHIYNVNEENPGFPEPRALFYTAQIICGLEHLHQRRIVYRDLKPENVLLDNDGNVRISDLGLAVELLDGQSKTKGYAGTPGFMAPELLQGEEYDFSVDYFALGVTLYEMIAARGPFRARGEKVENKELKHRIISEPVKYPDKFSQASKDFCEALLEKDPEKRLGFRDETCDKLRAHPLFKDLNWRQLEAGMLMPPFIPDSKTVYAKDIQDVGAFSTVKGVAFDKTDTEFFQEFATGNCPIPWQEEMIETGIFGELNVWRSDGQMPDDMKGISGGSSSSSKSGMCLVS",
# RIPK1_HUMAN
"MQPDMSLNVIKMKSSDFLESAELDSGGFGKVSLCFHRTQGLMIMKTVYKGPNCIEHNEALLEEAKMMNRLRHSRVVKLLGVIIEEGKYSLVMEYMEKGNLMHVLKAEMSTPLSVKGRIILEIIEGMCYLHGKGVIHKDLKPENILVDNDFHIKIADLGLASFKMWSKLNNEEHNELREVDGTAKKNGGTLYYMAPEHLNDVNAKPTEKSDVYSFAVVLWAIFANKEPYENAICEQQLIMCIKSGNRPDVDDITEYCPREIISLMKLCWEANPEARPTFPGIEEKFRPFYLSQLEESVEEDVKSLKKEYSNENAVVKRMQSLQLDCVAVPSSRSNSATEQPGSLHSSQGLGMGPVEESWFAPSLEHPQEENEPSLQSKLQDEANYHLYGSRMDRQTKQQPRQNVAYNREEERRRRVSHDPFAQQRPYENFQNTEGKGTAYSSAASHGNAVHQPSGLTSQPQVLYQNNGLYSSHGFGTRPLDPGTAGPRVWYRPIPSHMPSLHNIPVPETNYLGNTPTMPFSSLPPTDESIKYTIYNSTGIQIGAYNYMEIGGTSSSLLDSTNTNFKEEPAAKYQAIFDNTTSLTDKHLDPIRENLGKHWKNCARKLGFTQSQIDEIDHDYERDGLKEKVYQMLQKWVMREGIKGATVGKLAQALHQCSRIDLLSSLIYVSQN",
# RIPK2_HUMAN
"MNGEAICSALPTIPYHKLADLRYLSRGASGTVSSARHADWRVQVAVKHLHIHTPLLDSERKDVLREAEILHKARFSYILPILGICNEPEFLGIVTEYMPNGSLNELLHRKTEYPDVAWPLRFRILHEIALGVNYLHNMTPPLLHHDLKTQNILLDNEFHVKIADFGLSKWRMMSLSQSRSSKSAPEGGTIIYMPPENYEPGQKSRASIKHDIYSYAVITWEVLSRKQPFEDVTNPLQIMYSVSQGHRPVINEESLPYDIPHRARMISLIESGWAQNPDERPSFLKCLIELEPVLRTFEEITFLEAVIQLKKTKLQSVSSAIHLCDKKKMELSLNIPVNHGPQEESCGSSQLHENSGSPETSRSLPAPQDNDFLSRKAQDCYFMKLHHCPGNHSWDSTISGSQRAAFCDHKTTPCSSAIINPLSTAGNSERLQPGIAQQWIQSKREDIVNQMTEACLNQSLDALLSRDLIMKEDYELVSTKPTRTSKVRQLLDTTDIQGEEFAKVIVQKLKDNKQMGLQPYPEILVVSRSPSLNLLQNKSM",
# RIPK3_HUMAN
"MSCVKLWPSGAPAPLVSIEELENQELVGKGGFGTVFRAQHRKWGYDVAVKIVNSKAISREVKAMASLDNEFVLRLEGVIEKVNWDQDPKPALVTKFMENGSLSGLLQSQCPRPWPLLCRLLKEVVLGMFYLHDQNPVLLHRDLKPSNVLLDPELHVKLADFGLSTFQGGSQSGTGSGEPGGTLGYLAPELFVNVNRKASTASDVYSFGILMWAVLAGREVELPTEPSLVYEAVCNRQNRPSLAELPQAGPETPGLEGLKELMQLCWSSEPKDRPSFQECLPKTDEVFQMVENNMNAAVSTVKDFLSQLRSSNRRFSIPESGQGGTEMDGFRRTIENQHSRNDVMVSEWLNKLNLEEPPSSVPKKCPSLTKRSRAQEEQVPQAWTAGTSSDSMAQPPQTPETSTFRNQMPSPTSTGTPSPGPRGNQGAERQGMNWSCRTPEPNPVTGRPLVNIYNCSGVQVGDNNYLTMQQTTALPTWGLAPSGKGRGLQHPPPVGSQEGPKDPEAWSRPQGWYNHSGK",
# ROCK1_HUMAN
"MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPALRKNKNIDNFLSRYKDTINKIRDLRMKAEDYEVVKVIGRGAFGEVQLVRHKSTRKVYAMKLLSKFEMIKRSDSAFFWEERDIMAFANSPWVVQLFYAFQDDRYLYMVMEYMPGGDLVNLMSNYDVPEKWARFYTAEVVLALDAIHSMGFIHRDVKPDNMLLDKSGHLKLADFGTCMKMNKEGMVRCDTAVGTPDYISPEVLKSQGGDGYYGRECDWWSVGVFLYEMLVGDTPFYADSLVGTYSKIMNHKNSLTFPDDNDISKEAKNLICAFLTDREVRLGRNGVEEIKRHLFFKNDQWAWETLRDTVAPVVPDLSSDIDTSNFDDLEEDKGEEETFPIPKAFVGNQLPFVGFTYYSNRRYLSSANPNDNRTSSNADKSLQESLQKTIYKLEEQLHNEMQLKDEMEQKCRTSNIKLDKIMKELDEEGNQRRNLESTVSQIEKEKMLLQHRINEYQRKAEQENEKRRNVENEVSTLKDQLEDLKKVSQNSQLANEKLSQLQKQLEEANDLLRTESDTAVRLRKSHTEMSKSISQLESLNRELQERNRILENSKSQTDKDYYQLQAILEAERRDRGHDSEMIGDLQARITSLQEEVKHLKHNLEKVEGERKEAQDMLNHSEKEKNNLEIDLNYKLKSLQQRLEQEVNEHKVTKARLTDKHQSIEEAKSVAMCEMEKKLKEEREAREKAENRVVQIEKQCSMLDVDLKQSQQKLEHLTGNKERMEDEVKNLTLQLEQESNKRLLLQNELKTQAFEADNLKGLEKQMKQEINTLLEAKRLLEFELAQLTKQYRGNEGQMRELQDQLEAEQYFSTLYKTQVKELKEEIEEKNRENLKKIQELQNEKETLATQLDLAETKAESEQLARGLLEEQYFELTQESKKAASRNRQEITDKDHTVSRLEEANSMLTKDIEILRRENEELTEKMKKAEEEYKLEKEEEISNLKAAFEKNINTERTLKTQAVNKLAEIMNRKDFKIDRKKANTQDLRKKEKENRKLQLELNQEREKFNQMVVKHQKELNDMQAQLVEECAHRNELQMQLASKESDIEQLRAKLLDLSDSTSVASFPSADETDGNLPESRIEGWLSVPNRGNIKRYGWKKQYVVVSSKKILFYNDEQDKEQSNPSMVLDIDKLFHVRPVTQGDVYRAETEEIPKIFQILYANEGECRKDVEMEPVQQAEKTNFQNHKGHEFIPTLYHFPANCDACAKPLWHVFKPPPALECRRCHVKCHRDHLDKKEDLICPCKVSYDVTSARDMLLLACSQDEQKKWVTHLVKKIPKNPPSGFVRASPRTLSTRSTANQSFRKVVKNTSGKTS",
# RON_HUMAN
"MELLPPLPQSFLLLLLLPAKPAAGEDWQCPRTPYAASRDFDVKYVVPSFSAGGLVQAMVTYEGDRNESAVFVAIRNRLHVLGPDLKSVQSLATGPAGDPGCQTCAACGPGPHGPPGDTDTKVLVLDPALPALVSCGSSLQGRCFLHDLEPQGTAVHLAAPACLFSAHHNRPDDCPDCVASPLGTRVTVVEQGQASYFYVASSLDAAVAASFSPRSVSIRRLKADASGFAPGFVALSVLPKHLVSYSIEYVHSFHTGAFVYFLTVQPASVTDDPSALHTRLARLSATEPELGDYRELVLDCRFAPKRRRRGAPEGGQPYPVLRVAHSAPVGAQLATELSIAEGQEVLFGVFVTGKDGGPGVGPNSVVCAFPIDLLDTLIDEGVERCCESPVHPGLRRGLDFFQSPSFCPNPPGLEALSPNTSCRHFPLLVSSSFSRVDLFNGLLGPVQVTALYVTRLDNVTVAHMGTMDGRILQVELVRSLNYLLYVSNFSLGDSGQPVQRDVSRLGDHLLFASGDQVFQVPIQGPGCRHFLTCGRCLRAWHFMGCGWCGNMCGQQKECPGSWQQDHCPPKLTEFHPHSGPLRGSTRLTLCGSNFYLHPSGLVPEGTHQVTVGQSPCRPLPKDSSKLRPVPRKDFVEEFECELEPLGTQAVGPTNVSLTVTNMPPGKHFRVDGTSVLRGFSFMEPVLIAVQPLFGPRAGGTCLTLEGQSLSVGTSRAVLVNGTECLLARVSEGQLLCATPPGATVASVPLSLQVGGAQVPGSWTFQYREDPVVLSISPNCGYINSHITICGQHLTSAWHLVLSFHDGLRAVESRCERQLPEQQLCRLPEYVVRDPQGWVAGNLSARGDGAAGFTLPGFRFLPPPHPPSANLVPLKPEEHAIKFEYIGLGAVADCVGINVTVGGESCQHEFRGDMVVCPLPPSLQLGQDGAPLQVCVDGECHILGRVVRPGPDGVPQSTLLGILLPLLLLVAALATALVFSYWWRRKQLVLPPNLNDLASLDQTAGATPLPILYSGSDYRSGLALPAIDGLDSTTCVHGASFSDSEDESCVPLLRKESIQLRDLDSALLAEVKDVLIPHERVVTHSDRVIGKGHFGVVYHGEYIDQAQNRIQCAIKSLSRITEMQQVEAFLREGLLMRGLNHPNVLALIGIMLPPEGLPHVLLPYMCHGDLLQFIRSPQRNPTVKDLISFGLQVARGMEYLAEQKFVHRDLAARNCMLDESFTVKVADFGLARDILDREYYSVQQHRHARLPVKWMALESLQTYRFTTKSDVWSFGVLLWELLTRGAPPYRHIDPFDLTHFLAQGRRLPQPEYCPDSLYQVMQQCWEADPAVRPTFRVLVGEVEQIVSALLGDHYVQLPATYMNLGPSTSHEMNVRPEQPQFSPMPGNVRRPRPLSEPPRPT",
# ROR1_HUMAN
"MHRPRRRGTRPPLLALLAALLLAARGAAAQETELSVSAELVPTSSWNISSELNKDSYLTLDEPMNNITTSLGQTAELHCKVSGNPPPTIRWFKNDAPVVQEPRRLSFRSTIYGSRLRIRNLDTTDTGYFQCVATNGKEVVSSTGVLFVKFGPPPTASPGYSDEYEEDGFCQPYRGIACARFIGNRTVYMESLHMQGEIENQITAAFTMIGTSSHLSDKCSQFAIPSLCHYAFPYCDETSSVPKPRDLCRDECEILENVLCQTEYIFARSNPMILMRLKLPNCEDLPQPESPEAANCIRIGIPMADPINKNHKCYNSTGVDYRGTVSVTKSGRQCQPWNSQYPHTHTFTALRFPELNGGHSYCRNPGNQKEAPWCFTLDENFKSDLCDIPACDSKDSKEKNKMEILYILVPSVAIPLAIALLFFFICVCRNNQKSSSAPVQRQPKHVRGQNVEMSMLNAYKPKSKAKELPLSAVRFMEELGECAFGKIYKGHLYLPGMDHAQLVAIKTLKDYNNPQQWTEFQQEASLMAELHHPNIVCLLGAVTQEQPVCMLFEYINQGDLHEFLIMRSPHSDVGCSSDEDGTVKSSLDHGDFLHIAIQIAAGMEYLSSHFFVHKDLAARNILIGEQLHVKISDLGLSREIYSADYYRVQSKSLLPIRWMPPEAIMYGKFSSDSDIWSFGVVLWEIFSFGLQPYYGFSNQEVIEMVRKRQLLPCSEDCPPRMYSLMTECWNEIPSRRPRFKDIHVRLRSWEGLSSHTSSTTPSGGNATTQTTSLSASPVSNLSNPRYPNYMFPSQGITPQGQIAGFIGPPIPQNQRFIPINGYPIPPGYAAFPAAHYQPTGPPRVIQHCPPPKSRSPSSASGSTSTGHVTSLPSSGSNQEANIPLLPHMSIPNHPGGMGITVFGNKSQKPYKIDSKQASLLGDANIHGHTESMISAEL",
# ROR2_HUMAN
"MARGSALPRRPLLCIPAVWAAAALLLSVSRTSGEVEVLDPNDPLGPLDGQDGPIPTLKGYFLNFLEPVNNITIVQGQTAILHCKVAGNPPPNVRWLKNDAPVVQEPRRIIIRKTEYGSRLRIQDLDTTDTGYYQCVATNGMKTITATGVLFVRLGPTHSPNHNFQDDYHEDGFCQPYRGIACARFIGNRTIYVDSLQMQGEIENRITAAFTMIGTSTHLSDQCSQFAIPSFCHFVFPLCDARSRTPKPRELCRDECEVLESDLCRQEYTIARSNPLILMRLQLPKCEALPMPESPDAANCMRIGIPAERLGRYHQCYNGSGMDYRGTASTTKSGHQCQPWALQHPHSHHLSSTDFPELGGGHAYCRNPGGQMEGPWCFTQNKNVRMELCDVPSCSPRDSSKMGILYILVPSIAIPLVIACLFFLVCMCRNKQKASASTPQRRQLMASPSQDMEMPLINQHKQAKLKEISLSAVRFMEELGEDRFGKVYKGHLFGPAPGEQTQAVAIKTLKDKAEGPLREEFRHEAMLRARLQHPNVVCLLGVVTKDQPLSMIFSYCSHGDLHEFLVMRSPHSDVGSTDDDRTVKSALEPPDFVHLVAQIAAGMEYLSSHHVVHKDLATRNVLVYDKLNVKISDLGLFREVYAADYYKLLGNSLLPIRWMAPEAIMYGKFSIDSDIWSYGVVLWEVFSYGLQPYCGYSNQDVVEMIRNRQVLPCPDDCPAWVYALMIECWNEFPSRRPRFKDIHSRLRAWGNLSNYNSSAQTSGASNTTQTSSLSTSPVSNVSNARYVGPKQKAPPFPQPQFIPMKGQIRPMVPPPQLYVPVNGYQPVPAYGAYLPNFYPVQIPMQMAPQQVPPQMVPKPSSHHSGSGSTSTGYVTTAPSNTSMADRAALLSEGADDTQNAPEDGAQSTVQEAEEEEEGSVPETELLGDCDTLQVDEAQVQLEA",
# ROS1_HUMAN
"MKNIYCLIPKLVNFATLGCLWISVVQCTVLNSCLKSCVTNLGQQLDLGTPHNLSEPCIQGCHFWNSVDQKNCALKCRESCEVGCSSAEGAYEEEVLENADLPTAPFASSIGSHNMTLRWKSANFSGVKYIIQWKYAQLLGSWTYTKTVSRPSYVVKPLHPFTEYIFRVVWIFTAQLQLYSPPSPSYRTHPHGVPETAPLIRNIESSSPDTVEVSWDPPQFPGGPILGYNLRLISKNQKLDAGTQRTSFQFYSTLPNTIYRFSIAAVNEVGEGPEAESSITTSSSAVQQEEQWLFLSRKTSLRKRSLKHLVDEAHCLRLDAIYHNITGISVDVHQQIVYFSEGTLIWAKKAANMSDVSDLRIFYRGSGLISSISIDWLYQRMYFIMDELVCVCDLENCSNIEEITPPSISAPQKIVADSYNGYVFYLLRDGIYRADLPVPSGRCAEAVRIVESCTLKDFAIKPQAKRIIYFNDTAQVFMSTFLDGSASHLILPRIPFADVKSFACENNDFLVTDGKVIFQQDALSFNEFIVGCDLSHIEEFGFGNLVIFGSSSQLHPLPGRPQELSVLFGSHQALVQWKPPALAIGANVILISDIIELFELGPSAWQNWTYEVKVSTQDPPEVTHIFLNISGTMLNVPELQSAMKYKVSVRASSPKRPGPWSEPSVGTTLVPASEPPFIMAVKEDGLWSKPLNSFGPGEFLSSDIGNVSDMDWYNNSLYYSDTKGDVFVWLLNGTDISENYHLPSIAGAGALAFEWLGHFLYWAGKTYVIQRQSVLTGHTDIVTHVKLLVNDMVVDSVGGYLYWTTLYSVESTRLNGESSLVLQTQPWFSGKKVIALTLDLSDGLLYWLVQDSQCIHLYTAVLRGQSTGDTTITEFAAWSTSEISQNALMYYSGRLFWINGFRIITTQEIGQKTSVSVLEPARFNQFTIIQTSLKPLPGNFSFTPKVIPDSVQESSFRIEGNASSFQILWNGPPAVDWGVVFYSVEFSAHSKFLASEQHSLPVFTVEGLEPYALFNLSVTPYTYWGKGPKTSLSLRAPETVPSAPENPRIFILPSGKCCNKNEVVVEFRWNKPKHENGVLTKFEIFYNISNQSITNKTCEDWIAVNVTPSVMSFQLEGMSPRCFIAFQVRAFTSKGPGPYADVVKSTTSEINPFPHLITLLGNKIVFLDMDQNQVVWTFSAERVISAVCYTADNEMGYYAEGDSLFLLHLHNRSSSELFQDSLVFDITVITIDWISRHLYFALKESQNGMQVFDVDLEHKVKYPREVKIHNRNSTIISFSVYPLLSRLYWTEVSNFGYQMFYYSIISHTLHRILQPTATNQQNKRNQCSCNVTEFELSGAMAIDTSNLEKPLIYFAKAQEIWAMDLEGCQCWRVITVPAMLAGKTLVSLTVDGDLIYWIITAKDSTQIYQAKKGNGAIVSQVKALRSRHILAYSSVMQPFPDKAFLSLASDTVEPTILNATNTSLTIRLPLAKTNLTWYGITSPTPTYLVYYAEVNDRKNSSDLKYRILEFQDSIALIEDLQPFSTYMIQIAVKNYYSDPLEHLPPGKEIWGKTKNGVPEAVQLINTTVRSDTSLIISWRESHKPNGPKESVRYQLAISHLALIPETPLRQSEFPNGRLTLLVTRLSGGNIYVLKVLACHSEEMWCTESHPVTVEMFNTPEKPYSLVPENTSLQFNWKAPLNVNLIRFWVELQKWKYNEFYHVKTSCSQGPAYVCNITNLQPYTSYNVRVVVVYKTGENSTSLPESFKTKAGVPNKPGIPKLLEGSKNSIQWEKAEDNGCRITYYILEIRKSTSNNLQNQNLRWKMTFNGSCSSVCTWKSKNLKGIFQFRVVAANNLGFGEYSGISENIILVGDDFWIPETSFILTIIVGIFLVVTIPLTFVWHRRLKNQKSAKEGVTVLINEDKELAELRGLAAGVGLANACYAIHTLPTQEEIENLPAFPREKLTLRLLLGSGAFGEVYEGTAVDILGVGSGEIKVAVKTLKKGSTDQEKIEFLKEAHLMSKFNHPNILKQLGVCLLNEPQYIILELMEGGDLLTYLRKARMATFYGPLLTLVDLVDLCVDISKGCVYLERMHFIHRDLAARNCLVSVKDYTSPRIVKIGDFGLARDIYKNDYYRKRGEGLLPVRWMAPESLMDGIFTTQSDVWSFGILIWEILTLGHQPYPAHSNLDVLNYVQTGGRLEPPRNCPDDLWNLMTQCWAQEPDQRPTFHRIQDQLQLFRNFFLNSIYKSRDEANNSGVINESFEGEDGDVICLNSDDIMPVALMETKNREGLNYMVLATECGQGEEKSEGPLGSQESESCGLRKEEKEPHADKDFCQEKQVAYCPSGKPEGLNYACLTHSGYGDGSD",
# KS6A2_HUMAN
"MDLSMKKFAVRRFFSVYLRRKSRSKSSSLSRLEEEGVVKEIDISHHVKEGFEKADPSQFELLKVLGQGSYGKVFLVRKVKGSDAGQLYAMKVLKKATLKVRDRVRSKMERDILAEVNHPFIVKLHYAFQTEGKLYLILDFLRGGDLFTRLSKEVMFTEEDVKFYLAELALALDHLHSLGIIYRDLKPENILLDEEGHIKITDFGLSKEAIDHDKRAYSFCGTIEYMAPEVVNRRGHTQSADWWSFGVLMFEMLTGSLPFQGKDRKETMALILKAKLGMPQFLSGEAQSLLRALFKRNPCNRLGAGIDGVEEIKRHPFFVTIDWNTLYRKEIKPPFKPAVGRPEDTFHFDPEFTARTPTDSPGVPPSANAHHLFRGFSFVASSLIQEPSQQDLHKVPVHPIVQQLHGNNIHFTDGYEIKEDIGVGSYSVCKRCVHKATDTEYAVKIIDKSKRDPSEEIEILLRYGQHPNIITLKDVYDDGKFVYLVMELMRGGELLDRILRQRYFSEREASDVLCTITKTMDYLHSQGVVHRDLKPSNILYRDESGSPESIRVCDFGFAKQLRAGNGLLMTPCYTANFVAPEVLKRQGYDAACDIWSLGILLYTMLAGFTPFANGPDDTPEEILARIGSGKYALSGGNWDSISDAAKDVVSKMLHVDPHQRLTAMQVLKHPWVVNREYLSPNQLSRQDVHLVKGAMAATYFALNRTPQAPRLEPVLSSNLAQRRGMKRLTSTRL",
# KS6A3_HUMAN
"MPLAQLADPWQKMAVESPSDSAENGQQIMDEPMGEEEINPQTEEVSIKEIAITHHVKEGHEKADPSQFELLKVLGQGSFGKVFLVKKISGSDARQLYAMKVLKKATLKVRDRVRTKMERDILVEVNHPFIVKLHYAFQTEGKLYLILDFLRGGDLFTRLSKEVMFTEEDVKFYLAELALALDHLHSLGIIYRDLKPENILLDEEGHIKLTDFGLSKESIDHEKKAYSFCGTVEYMAPEVVNRRGHTQSADWWSFGVLMFEMLTGTLPFQGKDRKETMTMILKAKLGMPQFLSPEAQSLLRMLFKRNPANRLGAGPDGVEEIKRHSFFSTIDWNKLYRREIHPPFKPATGRPEDTFYFDPEFTAKTPKDSPGIPPSANAHQLFRGFSFVAITSDDESQAMQTVGVHSIVQQLHRNSIQFTDGYEVKEDIGVGSYSVCKRCIHKATNMEFAVKIIDKSKRDPTEEIEILLRYGQHPNIITLKDVYDDGKYVYVVTELMKGGELLDKILRQKFFSEREASAVLFTITKTVEYLHAQGVVHRDLKPSNILYVDESGNPESIRICDFGFAKQLRAENGLLMTPCYTANFVAPEVLKRQGYDAACDIWSLGVLLYTMLTGYTPFANGPDDTPEEILARIGSGKFSLSGGYWNSVSDTAKDLVSKMLHVDPHQRLTAALVLRHPWIVHWDQLPQYQLNRQDAPHLVKGAMAATYSALNRNQSPVLEPVGRSTLAQRRGIKKITSTAL",
# KS6A1_HUMAN
"MPLAQLKEPWPLMELVPLDPENGQTSGEEAGLQPSKDEGVLKEISITHHVKAGSEKADPSHFELLKVLGQGSFGKVFLVRKVTRPDSGHLYAMKVLKKATLKVRDRVRTKMERDILADVNHPFVVKLHYAFQTEGKLYLILDFLRGGDLFTRLSKEVMFTEEDVKFYLAELALGLDHLHSLGIIYRDLKPENILLDEEGHIKLTDFGLSKEAIDHEKKAYSFCGTVEYMAPEVVNRQGHSHSADWWSYGVLMFEMLTGSLPFQGKDRKETMTLILKAKLGMPQFLSTEAQSLLRALFKRNPANRLGSGPDGAEEIKRHVFYSTIDWNKLYRREIKPPFKPAVAQPDDTFYFDTEFTSRTPKDSPGIPPSAGAHQLFRGFSFVATGLMEDDGKPRAPQAPLHSVVQQLHGKNLVFSDGYVVKETIGVGSYSECKRCVHKATNMEYAVKVIDKSKRDPSEEIEILLRYGQHPNIITLKDVYDDGKHVYLVTELMRGGELLDKILRQKFFSEREASFVLHTIGKTVEYLHSQGVVHRDLKPSNILYVDESGNPECLRICDFGFAKQLRAENGLLMTPCYTANFVAPEVLKRQGYDEGCDIWSLGILLYTMLAGYTPFANGPSDTPEEILTRIGSGKFTLSGGNWNTVSETAKDLVSKMLHVDPHQRLTAKQVLQHPWVTQKDKLPQSQLSHQDLQLVKGAMAATYSALNSSKPTPQLKPIESSILAQRRVRKLPSTTL",
# RYK_HUMAN
"MRGAARLGRPGRSCLPGARGLRAPPPPPLLLLLALLPLLPAPGAAAAPAPRPPELQSASAGPSVSLYLSEDEVRRLIGLDAELYYVRNDLISHYALSFSLLVPSETNFLHFTWHAKSKVEYKLGFQVDNVLAMDMPQVNISVQGEVPRTLSVFRVELSCTGKVDSEVMILMQLNLTVNSSKNFTVLNFKRRKMCYKKLEEVKTSALDKNTSRTIYDPVHAAPTTSTRVFYISVGVCCAVIFLVAIILAVLHLHSMKRIELDDSISASSSSQGLSQPSTQTTQYLRADTPNNATPITSYPTLRIEKNDLRSVTLLEAKGKVKDIAISRERITLKDVLQEGTFGRIFHGILIDEKDPNKEKQAFVKTVKDQASEIQVTMMLTESCKLRGLHHRNLLPITHVCIEEGEKPMVILPYMNWGNLKLFLRQCKLVEANNPQAISQQDLVHMAIQIACGMSYLARREVIHKDLAARNCVIDDTLQVKITDNALSRDLFPMDYHCLGDNENRPVRWMALESLVNNEFSSASDVWAFGVTLWELMTLGQTPYVDIDPFEMAAYLKDGYRIAQPINCPDELFAVMACCWALDPEERPKFQQLVQCLTEFHAALGAYV",
# PLK4_HUMAN
"MATCIGEKIEDFKVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKKAMYKAGMVQRVQNEVKIHCQLKHPSILELYNYFEDSNYVYLVLEMCHNGEMNRYLKNRVKPFSENEARHFMHQIITGMLYLHSHGILHRDLTLSNLLLTRNMNIKIADFGLATQLKMPHEKHYTLCGTPNYISPEIATRSAHGLESDVWSLGCMFYTLLIGRPPFDTDTVKNTLNKVVLADYEMPSFLSIEAKDLIHQLLRRNPADRLSLSSVLDHPFMSRNSSTKSKDLGTVEDSIDSGHATISTAITASSSTSISGSLFDKRRLLIGQPLPNKMTVFPKNKSSTDFSSSGDGNSFYTQWGNQETSNSGRGRVIQDAEERPHSRYLRRAYSSDRSGTSNSQSQAKTYTMERCHSAEMLSVSKRSGGGENEERYSPTDNNANIFNFFKEKTSSSSGSFERPDNNQALSNHLCPGKTPFPFADPTPQTETVQQWFGNLQINAHLRKTTEYDSISPNRDFQGHPDLQKDTSKNAWTDTKVKKNSDASDNAHSVKQQNTMKYMTALHSKPEIIQQECVFGSDPLSEQSKTRGMEPPWGYQNRTLRSITSPLVAHRLKPIRQKTKKAVVSILDSEEVCVELVKEYASQEYVKEVLQISSDGNTITIYYPNGGRGFPLADRPPSPTDNISRYSFDNLPEKYWRKYQYASRFVQLVRSKSPKITYFTRYAKCILMENSPGADFEVWFYDGVKIHKTEDFIQVIEKTGKSYTLKSESEVNSLKEEIKMYMDHANEGHRICLALESIISEEERKTRSAPFFPIIIGRKPGSTSSPKALSPPPSVDSNYPTRERASFNRMVMHSAASPTQAPILNPSMVTNEGLGLTTTASGTDISSNSLKDCLPKSAQLLKSVFVKNVGWATQLTSGAVWVQFNDGSQLVVQAGVSSISYTSPNGQTTRYGENEKLPDYIKQKLQCLSSILLMFSNPTPNFH",
# MK11_HUMAN
"MSGPRAGFYRQELNKTVWEVPQRLQGLRPVGSGAYGSVCSAYDARLRQKVAVKKLSRPFQSLIHARRTYRELRLLKHLKHENVIGLLDVFTPATSIEDFSEVYLVTTLMGADLNNIVKCQALSDEHVQFLVYQLLRGLKYIHSAGIIHRDLKPSNVAVNEDCELRILDFGLARQADEEMTGYVATRWYRAPEIMLNWMHYNQTVDIWSVGCIMAELLQGKALFPGSDYIDQLKRIMEVVGTPSPEVLAKISSEHARTYIQSLPPMPQKDLSSIFRGANPLAIDLLGRMLVLDSDQRVSAAEALAHAYFSQYHDPEDEPEAEPYDESVEAKERTLEEWKELTYQEVLSFKPPEPPKPPGSLEIEQ",
# MK12_HUMAN
"MSSPPPARSGFYRQEVTKTAWEVRAVYRDLQPVGSGAYGAVCSAVDGRTGAKVAIKKLYRPFQSELFAKRAYRELRLLKHMRHENVIGLLDVFTPDETLDDFTDFYLVMPFMGTDLGKLMKHEKLGEDRIQFLVYQMLKGLRYIHAAGIIHRDLKPGNLAVNEDCELKILDFGLARQADSEMTGYVVTRWYRAPEVILNWMRYTQTVDIWSVGCIMAEMITGKTLFKGSDHLDQLKEIMKVTGTPPAEFVQRLQSDEAKNYMKGLPELEKKDFASILTNASPLAVNLLEKMLVLDAEQRVTAGEALAHPYFESLHDTEDEPQVQKYDDSFDDVDRTLDEWKRVTYKEVLSFKPPRQLGARVSKETPL",
# MK13_HUMAN
"MSLIRKKGFYKQDVNKTAWELPKTYVSPTHVGSGAYGSVCSAIDKRSGEKVAIKKLSRPFQSEIFAKRAYRELLLLKHMQHENVIGLLDVFTPASSLRNFYDFYLVMPFMQTDLQKIMGMEFSEEKIQYLVYQMLKGLKYIHSAGVVHRDLKPGNLAVNEDCELKILDFGLARHADAEMTGYVVTRWYRAPEVILSWMHYNQTVDIWSVGCIMAEMLTGKTLFKGKDYLDQLTQILKVTGVPGTEFVQKLNDKAAKSYIQSLPQTPRKDFTQLFPRASPQAADLLEKMLELDVDKRLTAAQALTHPFFEPFRDPEEETEAQQPFDDSLEHEKLTVDEWKQHIYKEIVNFSPIARKDSRRRSGMKL",
# MAST1_HUMAN
"MSDSLWTALSNFSMPSFPGGSMFRRTKSCRTSNRKSLILTSTSPTLPRPHSPLPGHLGSSPLDSPRNFSPNTPAHFSFASSRRADGRRWSLASLPSSGYGTNTPSSTVSSSCSSQERLHQLPYQPTVDELHFLSKHFGSTESITDEDGGRRSPAVRPRSRSLSPGRSPSSYDNEIVMMNHVYKERFPKATAQMEEKLRDFTRAYEPDSVLPLADGVLSFIHHQIIELARDCLTKSRDGLITTVYFYELQENLEKLLQDAYERSESLEVAFVTQLVKKLLIIISRPARLLECLEFNPEEFYHLLEAAEGHAKEGHLVKTDIPRYIIRQLGLTRDPFPDVVHLEEQDSGGSNTPEQDDLSEGRSSKAKKPPGENDFDTIKLISNGAYGAVYLVRHRDTRQRFAMKKINKQNLILRNQIQQAFVERDILTFAENPFVVGMFCSFETRRHLCMVMEYVEGGDCATLLKNIGALPVEMARMYFAETVLALEYLHNYGIVHRDLKPDNLLITSMGHIKLTDFGLSKMGLMSLTTNLYEGHIEKDAREFLDKQVCGTPEYIAPEVILRQGYGKPVDWWAMGIILYEFLVGCVPFFGDTPEELFGQVISDDILWPEGDEALPTEAQLLISSLLQTNPLVRLGAGGAFEVKQHSFFRDLDWTGLLRQKAEFIPHLESEDDTSYFDTRSDRYHHVNSYDEDDTTEEEPVEIRQFSSCSPRFSKVYSSMEQLSQHEPKTPVAAAGSSKREPSTKGPEEKVAGKREGLGGLTLREKTWRGGSPEIKRFSASEASFLEGEASPPLGARRRFSALLEPSRFSAPQEDEDEARLRRPPRPSSDPAGSLDARAPKEETQGEGTSSAGDSEATDRPRPGDLCPPSKDGDASGPRATNDLVLRRARHQQMSGDVAVEKRPSRTGGKVIKSASATALSVMIPAVDPHGSSPLASPMSPRSLSSNPSSRDSSPSRDYSPAVSGLRSPITIQRSGKKYGFTLRAIRVYMGDTDVYSVHHIVWHVEEGGPAQEAGLCAGDLITHVNGEPVHGMVHPEVVELILKSGNKVAVTTTPFENTSIRIGPARRSSYKAKMARRNKRPSAKEGQESKKRSSLFRKITKQSNLLHTSRSLSSLNRSLSSSDSLPGSPTHGLPARSPTHSYRSTPDSAYLGASSQSSSPASSTPNSPASSASHHIRPSTLHGLSPKLHRQYRSARCKSAGNIPLSPLAHTPSPTQASPPPLPGHTVGSSHTTQSFPAKLHSSPPVVRPRPKSAEPPRSPLLKRVQSAEKLGASLSADKKGALRKHSLEVGHPDFRKDFHGELALHSLAESDGETPPVEGLGAPRQVAVRRLGRQESPLSLGADPLLPEGASRPPVSSKEKESPGGAEACTPPRATTPGGRTLERDVGCTRHQSVQTEDGTGGMARAVAKAALSPVQEHETGRRSSSGEAGTPLVPIVVEPARPGAKAVVPQPLGADSKGLQEPAPLAPSVPEAPRGRERWVLEVVEERTTLSGPRSKPASPKLSPEPQTPSLAPAKCSAPSSAVTPVPPASLLGSGTKPQVGLTSRCPAEAVPPAGLTKKGVSSPAPPGP",
# SGK1_HUMAN
"MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNSYACKHPEVQSILKISQPQEPELMNANPSPPPSPSQQINLGPSSNPHAKPSDFHFLKVIGKGSFGKVLLARHKAEEVFYAVKVLQKKAILKKKEEKHIMSERNVLLKNVKHPFLVGLHFSFQTADKLYFVLDYINGGELFYHLQRERCFLEPRARFYAAEIASALGYLHSLNIVYRDLKPENILLDSQGHIVLTDFGLCKENIEHNSTTSTFCGTPEYLAPEVLHKQPYDRTVDWWCLGAVLYEMLYGLPPFYSRNTAEMYDNILNKPLQLKPNITNSARHLLEGLLQKDRTKRLGAKDDFMEIKSHVFFSLINWDDLINKKITPPFNPNVSGPNDLRHFDPEFTEEPVPNSIGKSPDSVLVTASVKEAAEAFLGFSYAPPTDSFL",
# SLK_HUMAN
"MSFFNFRKIFKLGSEKKKKQYEHVKRDLNPEDFWEIIGELGDGAFGKVYKAQNKETSVLAAAKVIDTKSEEELEDYMVEIDILASCDHPNIVKLLDAFYYENNLWILIEFCAGGAVDAVMLELERPLTESQIQVVCKQTLDALNYLHDNKIIHRDLKAGNILFTLDGDIKLADFGVSAKNTRTIQRRDSFIGTPYWMAPEVVMCETSKDRPYDYKADVWSLGITLIEMAEIEPPHHELNPMRVLLKIAKSEPPTLAQPSRWSSNFKDFLKKCLEKNVDARWTTSQLLQHPFVTVDSNKPIRELIAEAKAEVTEEVEDGKEEDEEEETENSLPIPASKRASSDLSIASSEEDKLSQNACILESVSEKTERSNSEDKLNSKILNEKPTTDEPEKAVEDINEHITDAQLEAMTELHDRTAVIKENEREKRPKLENLPDTEDQETVDINSVSEGKENNIMITLETNIEHNLKSEEEKDQEKQQMFENKLIKSEEIKDTILQTVDLVSQETGEKEANIQAVDSEVGLTKEDTQEKLGEDDKTQKDVISNTSDVIGTCEAADVAQKVDEDSAEDTQSNDGKEVVEVGQKLINKPMVGPEAGGTKEVPIKEIVEMNEIEEGKNKEQAINSSENIMDINEEPGTTEGEEITESSSTEEMEVRSVVADTDQKALGSEVQDASKVTTQIDKEKKEIPVSIKKEPEVTVVSQPTEPQPVLIPSININSDSGENKEEIGSLSKTETILPPESENPKENDNDSGTGSTADTSSIDLNLSISSFLSKTKDSGSISLQETRRQKKTLKKTRKFIVDGVEVSVTTSKIVTDSDSKTEELRFLRRQELRELRFLQKEEQRAQQQLNSKLQQQREQIFRRFEQEMMSKKRQYDQEIENLEKQQKQTIERLEQEHTNRLRDEAKRIKGEQEKELSKFQNMLKNRKKEVINEVEKAPKELRKELMKRRKEELAQSQHAQEQEFVQKQQQELDGSLKKIIQQQKAELANIERECLNNKQQLMRAREAAIWELEERHLQEKHQLLKQQLKDQYFMQRHQLLKRHEKETEQMQRYNQRLIEELKNRQTQERARLPKIQRSEAKTRMAMFKKSLRINSTATPDQDRDKIKQFAAQEEKRQKNERMAQHQKHENQMRDLQLQCEANVRELHQLQNEKCHLLVEHETQKLKELDEEHSQELKEWREKLRPRKKTLEEEFARKLQEQEVFFKMTGESECLNPSTQSRISKFYPIPSLHSTGS",
# PLK2_HUMAN
"MELLRTITYQPAASTKMCEQALGKGCGADSKKKRPPQPPEESQPPQSQAQVPPAAPHHHHHHSHSGPEISRIIVDPTTGKRYCRGKVLGKGGFAKCYEMTDLTNNKVYAAKIIPHSRVAKPHQREKIDKEIELHRILHHKHVVQFYHYFEDKENIYILLEYCSRRSMAHILKARKVLTEPEVRYYLRQIVSGLKYLHEQEILHRDLKLGNFFINEAMELKVGDFGLAARLEPLEHRRRTICGTPNYLSPEVLNKQGHGCESDIWALGCVMYTMLLGRPPFETTNLKETYRCIREARYTMPSSLLAPAKHLIASMLSKNPEDRPSLDDIIRHDFFLQGFTPDRLSSSCCHTVPDFHLSSPAKNFFKKAAAALFGGKKDKARYIDTHNRVSKEDEDIYKLRHDLKKTSITQQPSKHRTDEELQPPTTTVARSGTPAVENKQQIGDAIRMIVRGTLGSCSSSSECLEDSTMGSVADTVARVLRGCLENMPEADCIPKEQLSTSFQWVTKWVDYSNKYGFGYQLSDHTVGVLFNNGAHMSLLPDKKTVHYYAELGQCSVFPATDAPEQFISQVTVLKYFSHYMEENLMDGGDLPSVTDIRRPRLYLLQWLKSDKALMMLFNDGTFQVNFYHDHTKIIICSQNEEYLLTYINEDRISTTFRLTTLLMSGCSSELKNRMEYALNMLLQRCN",
# M3K11_HUMAN
"MEPLKSLFLKSPLGSWNGSGSGGGGGGGGGRPEGSPKAAGYANPVWTALFDYEPSGQDELALRKGDRVEVLSRDAAISGDEGWWAGQVGGQVGIFPSNYVSRGGGPPPCEVASFQELRLEEVIGIGGFGKVYRGSWRGELVAVKAARQDPDEDISVTAESVRQEARLFAMLAHPNIIALKAVCLEEPNLCLVMEYAAGGPLSRALAGRRVPPHVLVNWAVQIARGMHYLHCEALVPVIHRDLKSNNILLLQPIESDDMEHKTLKITDFGLAREWHKTTQMSAAGTYAWMAPEVIKASTFSKGSDVWSFGVLLWELLTGEVPYRGIDCLAVAYGVAVNKLTLPIPSTCPEPFAQLMADCWAQDPHRRPDFASILQQLEALEAQVLREMPRDSFHSMQEGWKREIQGLFDELRAKEKELLSREEELTRAAREQRSQAEQLRRREHLLAQWELEVFERELTLLLQQVDRERPHVRRRRGTFKRSKLRARDGGERISMPLDFKHRITVQASPGLDRRRNVFEVGPGDSPTFPRFRAIQLEPAEPGQAWGRQSPRRLEDSSNGERRACWAWGPSSPKPGEAQNGRRRSRMDEATWYLDSDDSSPLGSPSTPPALNGNPPRPSLEPEEPKRPVPAERGSSSGTPKLIQRALLRGTALLASLGLGRDLQPPGGPGRERGESPTTPPTPTPAPCPTEPPPSPLICFSLKTPDSPPTPAPLLLDLGIPVGQRSAKSPRREEEPRGGTVSPPPGTSRSAPGTPGTPRSPPLGLISRPRPSPLRSRIDPWSFVSAGPRPSPLPSPQPAPRRAPWTLFPDSDPFWDSPPANPFQGGPQDCRAQTKDMGAQAPWVPEAGP",
# SRC_HUMAN
"MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAEPKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRESETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGLCHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTLKPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKYLRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVERGYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL",
# SRPK1_HUMAN
"MERKVLALQARKKRTKAKKDKAQRKSETQHRGSAPHSESDLPEQEEEILGSDDDEQEDPNDYCKGGYHLVKIGDLFNGRYHVIRKLGWGHFSTVWLSWDIQGKKFVAMKVVKSAEHYTETALDEIRLLKSVRNSDPNDPNREMVVQLLDDFKISGVNGTHICMVFEVLGHHLLKWIIKSNYQGLPLPCVKKIIQQVLQGLDYLHTKCRIIHTDIKPENILLSVNEQYIRRLAAEATEWQRSGAPPPSGSAVSTAPQPKPADKMSKNKKKKLKKKQKRQAELLEKRMQEIEEMEKESGPGQKRPNKQEESESPVERPLKENPPNKMTQEKLEESSTIGQDQTLMERDTEGGAAEINCNGVIEVINYTQNSNNETLRHKEDLHNANDCDVQNLNQESSFLSSQNGDSSTSQETDSCTPITSEVSDTMVCQSSSTVGQSFSEQHISQLQESIRAEIPCEDEQEQEHNGPLDNKGKSTAGNFLVNPLEPKNAEKLKVKIADLGNACWVHKHFTEDIQTRQYRSLEVLIGSGYNTPADIWSTACMAFELATGDYLFEPHSGEEYTRDEDHIALIIELLGKVPRKLIVAGKYSKEFFTKKGDLKHITKLKPWGLFEVLVEKYEWSQEEAAGFTDFLLPMLELIPEKRATAAECLRHPWLNS",
# SRPK2_HUMAN
"MSVNSEKSSSSERPEPQQKAPLVPPPPPPPPPPPPPLPDPTPPEPEEEILGSDDEEQEDPADYCKGGYHPVKIGDLFNGRYHVIRKLGWGHFSTVWLCWDMQGKRFVAMKVVKSAQHYTETALDEIKLLKCVRESDPSDPNKDMVVQLIDDFKISGMNGIHVCMVFEVLGHHLLKWIIKSNYQGLPVRCVKSIIRQVLQGLDYLHSKCKIIHTDIKPENILMCVDDAYVRRMAAEATEWQKAGAPPPSGSAVSTAPQQKPIGKISKNKKKKLKKKQKRQAELLEKRLQEIEELEREAERKIIEENITSAAPSNDQDGEYCPEVKLKTTGLEEAAEAETAKDNGEAEDQEEKEDAEKENIEKDEDDVDQELANIDPTWIESPKTNGHIENGPFSLEQQLDDEDDDEEDCPNPEEYNLDEPNAESDYTYSSSYEQFNGELPNGRHKIPESQFPEFSTSLFSGSLEPVACGSVLSEGSPLTEQEESSPSHDRSRTVSASSTGDLPKAKTRAADLLVNPLDPRNADKIRVKIADLGNACWVHKHFTEDIQTRQYRSIEVLIGAGYSTPADIWSTACMAFELATGDYLFEPHSGEDYSRDEDHIAHIIELLGSIPRHFALSGKYSREFFNRRGELRHITKLKPWSLFDVLVEKYGWPHEDAAQFTDFLIPMLEMVPEKRASAGECLRHPWLNS",
# CDKL5_HUMAN
"MKIPNIGNVMNKFEILGVVGEGAYGVVLKCRHKETHEIVAIKKFKDSEENEEVKETTLRELKMLRTLKQENIVELKEAFRRRGKLYLVFEYVEKNMLELLEEMPNGVPPEKVKSYIYQLIKAIHWCHKNDIVHRDIKPENLLISHNDVLKLCDFGFARNLSEGNNANYTEYVATRWYRSPELLLGAPYGKSVDMWSVGCILGELSDGQPLFPGESEIDQLFTIQKVLGPLPSEQMKLFYSNPRFHGLRFPAVNHPQSLERRYLGILNSVLLDLMKNLLKLDPADRYLTEQCLNHPTFQTQRLLDRSPSRSAKRKPYHVESSTLSNRNQAGKSTALQSHHRSNSKDIQNLSVGLPRADEGLPANESFLNGNLAGASLSPLHTKTYQASSQPGSTSKDLTNNNIPHLLSPKEAKSKTEFDFNIDPKPSEGPGTKYLKSNSRSQQNRHSFMESSQSKAGTLQPNEKQSRHSYIDTIPQSSRSPSYRTKAKSHGALSDSKSVSNLSEARAQIAEPSTSRYFPSSCLDLNSPTSPTPTRHSDTRTLLSPSGRNNRNEGTLDSRRTTTRHSKTMEELKLPEHMDSSHSHSLSAPHESFSYGLGYTSPFSSQQRPHRHSMYVTRDKVRAKGLDGSLSIGQGMAARANSLQLLSPQPGEQLPPEMTVARSSVKETSREGTSSFHTRQKSEGGVYHDPHSDDGTAPKENRHLYNDPVPRRVGSFYRVPSPRPDNSFHENNVSTRVSSLPSESSSGTNHSKRQPAFDPWKSPENISHSEQLKEKEKQGFFRSMKKKKKKSQTVPNSDSPDLLTLQKSIHSASTPSSRPKEWRPEKISDLQTQSQPLKSLRKLLHLSSASNHPASSDPRFQPLTAQQTKNSFSEIRIHPLSQASGGSSNIRQEPAPKGRPALQLPGQMDPGWHVSSVTRSATEGPSYSEQLGAKSGPNGHPYNRTNRSRMPNLNDLKETAL",
# TAOK2_HUMAN
"MPAGGRAGSLKDPDVAELFFKDDPEKLFSDLREIGHGSFGAVYFARDVRNSEVVAIKKMSYSGKQSNEKWQDIIKEVRFLQKLRHPNTIQYRGCYLREHTAWLVMEYCLGSASDLLEVHKKPLQEVEIAAVTHGALQGLAYLHSHNMIHRDVKAGNILLSEPGLVKLGDFGSASIMAPANSFVGTPYWMAPEVILAMDEGQYDGKVDVWSLGITCIELAERKPPLFNMNAMSALYHIAQNESPVLQSGHWSEYFRNFVDSCLQKIPQDRPTSEVLLKHRFVLRERPPTVIMDLIQRTKDAVRELDNLQYRKMKKILFQEAPNGPGAEAPEEEEEAEPYMHRAGTLTSLESSHSVPSMSISASSQSSSVNSLADASDNEEEEEEEEEEEEEEEGPEAREMAMMQEGEHTVTSHSSIIHRLPGSDNLYDDPYQPEITPSPLQPPAAPAPTSTTSSARRRAYCRNRDHFATIRTASLVSRQIQEHEQDSALREQLSGYKRMRRQHQKQLLALESRLRGEREEHSARLQRELEAQRAGFGAEAEKLARRHQAIGEKEARAAQAEERKFQQHILGQQKKELAALLEAQKRTYKLRKEQLKEELQENPSTPKREKAEWLLRQKEQLQQCQAEEEAGLLRRQRQYFELQCRQYKRKMLLARHSLDQDLLREDLNKKQTQKDLECALLLRQHEATRELELRQLQAVQRTRAELTRLQHQTELGNQLEYNKRREQELRQKHAAQVRQQPKSLKVRAGQRPPGLPLPIPGALGPPNTGTPIEQQPCSPGQEAVLDQRMLGEEEEAVGERRILGKEGATLEPKQQRILGEESGAPSPSPQKHGSLVDEEVWGLPEEIEELRVPSLVPQERSIVGQEEAGTWSLWGKEDESLLDEEFELGWVQGPALTPVPEEEEEEEEGAPIGTPRDPGDGCPSPDIPPEPPPTHLRPCPASQLPGLLSHGLLAGLSFAVGSSSGLLPLLLLLLLPLLAAQGGGGLQAALLALEVGLVGLGASYLLLCTALHLPSSLFLLLAQGTALGAVLGLSWRRGLMGVPLGLGAAWLLAWPGLALPLVAMAAGGRWVRQQGPRVRRGISRLWLRVLLRLSPMAFRALQGCGAVGDRGLFALYPKTNKDGFRSRLPVPGPRRRNPRTTQHPLALLARVWVLCKGWNWRLARASQGLASHLPPWAIHTLASWGLLRGERPTRIPRLLPRSQRQLGPPASRQPLPGTLAGRRSRTRQSRALPPWR",
# KSYK_HUMAN
"MASSGMADSANHLPFFFGNITREEAEDYLVQGGMSDGLYLLRQSRNYLGGFALSVAHGRKAHHYTIERELNGTYAIAGGRTHASPADLCHYHSQESDGLVCLLKKPFNRPQGVQPKTGPFEDLKENLIREYVKQTWNLQGQALEQAIISQKPQLEKLIATTAHEKMPWFHGKISREESEQIVLIGSKTNGKFLIRARDNNGSYALCLLHEGKVLHYRIDKDKTGKLSIPEGKKFDTLWQLVEHYSYKADGLLRVLTVPCQKIGTQGNVNFGGRPQLPGSHPATWSAGGIISRIKSYSFPKPGHRKSSPAQGNRQESTVSFNPYEPELAPWAADKGPQREALPMDTEVYESPYADPEEIRPKEVYLDRKLLTLEDKELGSGNFGTVKKGYYQMKKVVKTVAVKILKNEANDPALKDELLAEANVMQQLDNPYIVRMIGICEAESWMLVMEMAELGPLNKYLQQNRHVKDKNIIELVHQVSMGMKYLEESNFVHRDLAARNVLLVTQHYAKISDFGLSKALRADENYYKAQTHGKWPVKWYAPECINYYKFSSKSDVWSFGVLMWEAFSYGQKPYRGMKGSEVTAMLEKGERMGCPAGCPREMYDLMNLCWTYDVENRPGFAAVELRLRNYYYDVVN",
# M3K7_HUMAN
"MSTASAASSSSSSSAGEMIEAPSQVLNFEEIDYKEIEVEEVVGRGAFGVVCKAKWRAKDVAIKQIESESERKAFIVELRQLSRVNHPNIVKLYGACLNPVCLVMEYAEGGSLYNVLHGAEPLPYYTAAHAMSWCLQCSQGVAYLHSMQPKALIHRDLKPPNLLLVAGGTVLKICDFGTACDIQTHMTNNKGSAAWMAPEVFEGSNYSEKCDVFSWGIILWEVITRRKPFDEIGGPAFRIMWAVHNGTRPPLIKNLPKPIESLMTRCWSKDPSQRPSMEEIVKIMTHLMRYFPGADEPLQYPCQYSDEGQSNSATSTGSFMDIASTNTSNKSDTNMEQVPATNDTIKRLESKLLKNQAKQQSESGRLSLGASRGSSVESLPPTSEGKRMSADMSEIEARIAATTAYSKPKRGHRKTASFGNILDVPEIVISGNGQPRRRSIQDLTVTGTEPGQVSSRSSSPSVRMITTSGPTSEKPTRSHPWTPDDSTDTNGSDNSIPMAYLTLDHQLQPLAPCPNSKESMAVFEQHCKMAQEYMKVQTEIALLLQRKQELVAELDQDEKDQQNTSRLVQEHKKLLDENKSLSTYYQQCKKQLEVIRSQQQKRQGTS",
# BMPR2_HUMAN
"MTSSLQRPWRVPWLPWTILLVSTAAASQNQERLCAFKDPYQQDLGIGESRISHENGTILCSKGSTCYGLWEKSKGDINLVKQGCWSHIGDPQECHYEECVVTTTPPSIQNGTYRFCCCSTDLCNVNFTENFPPPDTTPLSPPHSFNRDETIIIALASVSVLAVLIVALCFGYRMLTGDRKQGLHSMNMMEAAASEPSLDLDNLKLLELIGRGRYGAVYKGSLDERPVAVKVFSFANRQNFINEKNIYRVPLMEHDNIARFIVGDERVTADGRMEYLLVMEYYPNGSLCKYLSLHTSDWVSSCRLAHSVTRGLAYLHTELPRGDHYKPAISHRDLNSRNVLVKNDGTCVISDFGLSMRLTGNRLVRPGEEDNAAISEVGTIRYMAPEVLEGAVNLRDCESALKQVDMYALGLIYWEIFMRCTDLFPGESVPEYQMAFQTEVGNHPTFEDMQVLVSREKQRPKFPEAWKENSLAVRSLKETIEDCWDQDAEARLTAQCAEERMAELMMIWERNKSVSPTVNPMSTAMQNERNLSHNRRVPKIGPYPDYSSSSYIEDSIHHTDSIVKNISSEHSMSSTPLTIGEKNRNSINYERQQAQARIPSPETSVTSLSTNTTTTNTTGLTPSTGMTTISEMPYPDETNLHTTNVAQSIGPTPVCLQLTEEDLETNKLDPKEVDKNLKESSDENLMEHSLKQFSGPDPLSSTSSSLLYPLIKLAVEATGQQDFTQTANGQACLIPDVLPTQIYPLPKQQNLPKRPTSLPLNTKNSTKEPRLKFGSKHKSNLKQVETGVAKMNTINAAEPHVVTVTMNGVAGRNHSVNSHAATTQYANGTVLSGQTTNIVTHRAQEMLQNQFIGEDTRLNINSSPDEHEPLLRREQQAGHDEGVLDRLVDRRERPLEGGRTNSNNNNSNPCSEQDVLAQGVPSTAADPGPSKPRRAQRPNSLDLSATNVLDGSSIQIGESTQDGKSGSGEKIKKRVKTPYSLKRWRPSTWVISTESLDCEVNNNGSNRAVHSKSSTAVYLAEGGTATTMVSKDIGMNCL",
# TEC_HUMAN
"MNFNTILEEILIKRSQQKKKTSPLNYKERLFVLTKSMLTYYEGRAEKKYRKGFIDVSKIKCVEIVKNDDGVIPCQNKYPFQVVHDANTLYIFAPSPQSRDLWVKKLKEEIKNNNNIMIKYHPKFWTDGSYQCCRQTEKLAPGCEKYNLFESSIRKALPPAPETKKRRPPPPIPLEEEDNSEEIVVAMYDFQAAEGHDLRLERGQEYLILEKNDVHWWRARDKYGNEGYIPSNYVTGKKSNNLDQYEWYCRNMNRSKAEQLLRSEDKEGGFMVRDSSQPGLYTVSLYTKFGGEGSSGFRHYHIKETTTSPKKYYLAEKHAFGSIPEIIEYHKHNAAGLVTRLRYPVSVKGKNAPTTAGFSYEKWEINPSELTFMRELGSGLFGVVRLGKWRAQYKVAIKAIREGAMCEEDFIEEAKVMMKLTHPKLVQLYGVCTQQKPIYIVTEFMERGCLLNFLRQRQGHFSRDVLLSMCQDVCEGMEYLERNSFIHRDLAARNCLVSEAGVVKVSDFGMARYVLDDQYTSSSGAKFPVKWCPPEVFNYSRFSSKSDVWSFGVLMWEVFTEGRMPFEKYTNYEVVTMVTRGHRLYQPKLASNYVYEVMLRCWQEKPEGRPSFEDLLRTIDELVECEETFGR",
# TIE2_HUMAN
"MDSLASLVLCGVSLLLSGTVEGAMDLILINSLPLVSDAETSLTCIASGWRPHEPITIGRDFEALMNQHQDPLEVTQDVTREWAKKVVWKREKASKINGAYFCEGRVRGEAIRIRTMKMRQQASFLPATLTMTVDKGDNVNISFKKVLIKEEDAVIYKNGSFIHSVPRHEVPDILEVHLPHAQPQDAGVYSARYIGGNLFTSAFTRLIVRRCEAQKWGPECNHLCTACMNNGVCHEDTGECICPPGFMGRTCEKACELHTFGRTCKERCSGQEGCKSYVFCLPDPYGCSCATGWKGLQCNEACHPGFYGPDCKLRCSCNNGEMCDRFQGCLCSPGWQGLQCEREGIQRMTPKIVDLPDHIEVNSGKFNPICKASGWPLPTNEEMTLVKPDGTVLHPKDFNHTDHFSVAIFTIHRILPPDSGVWVCSVNTVAGMVEKPFNISVKVLPKPLNAPNVIDTGHNFAVINISSEPYFGDGPIKSKKLLYKPVNHYEAWQHIQVTNEIVTLNYLEPRTEYELCVQLVRRGEGGEGHPGPVRRFTTASIGLPPPRGLNLLPKSQTTLNLTWQPIFPSSEDDFYVEVERRSVQKSDQQNIKVPGNLTSVLLNNLHPREQYVVRARVNTKAQGEWSEDLTAWTLSDILPPQPENIKISNITHSSAVISWTILDGYSISSITIRYKVQGKNEDQHVDVKIKNATITQYQLKGLEPETAYQVDIFAENNIGSSNPAFSHELVTLPESQAPADLGGGKMLLIAILGSAGMTCLTVLLAFLIILQLKRANVQRRMAQAFQNVREEPAVQFNSGTLALNRKVKNNPDPTIYPVLDWNDIKFQDVIGEGNFGQVLKARIKKDGLRMDAAIKRMKEYASKDDHRDFAGELEVLCKLGHHPNIINLLGACEHRGYLYLAIEYAPHGNLLDFLRKSRVLETDPAFAIANSTASTLSSQQLLHFAADVARGMDYLSQKQFIHRDLAARNILVGENYVAKIADFGLSRGQEVYVKKTMGRLPVRWMAIESLNYSVYTTNSDVWSYGVLLWEIVSLGGTPYCGMTCAELYEKLPQGYRLEKPLNCDDEVYDLMRQCWREKPYERPSFAQILVSLNRMLEERKTYVNTTLYEKFTYAGIDCSAEEAA",
# TESK1_HUMAN
"MAGERPPLRGPGPGPGEVPGEGPPGPGGTGGGPGRGRPSSYRALRSAVSSLARVDDFHCAEKIGAGFFSEVYKVRHRQSGQVMVLKMNKLPSNRGNTLREVQLMNRLRHPNILRFMGVCVHQGQLHALTEYMNGGTLEQLLSSPEPLSWPVRLHLALDIARGLRYLHSKGVFHRDLTSKNCLVRREDRGFTAVVGDFGLAEKIPVYREGARKEPLAVVGSPYWMAPEVLRGELYDEKADVFAFGIVLCELIARVPADPDYLPRTEDFGLDVPAFRTLVGDDCPLPFLLLAIHCCNLEPSTRAPFTEITQHLEWILEQLPEPAPLTRTALTHNQGSVARGGPSATLPRPDPRLSRSRSDLFLPPSPESPPNWGDNLTRVNPFSLREDLRGGKIKLLDTPSKPVLPLVPPSPFPSTQLPLVTTPETLVQPGTPARRCRSLPSSPELPRRMETALPGPGPPAVGPSAEEKMECEGSSPEPEPPGPAPQLPLAVATDNFISTCSSASQPWSPRSGPVLNNNPPAVVVNSPQGWAGEPWNRAQHSLPRAAALERTEPSPPPSAPREPDEGLPCPGCCLGPFSFGFLSMCPRPTPAVARYRNLNCEAGSLLCHRGHHAKPPTPSLQLPGARS",
# TGFR2_HUMAN
"MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVKFPQLCKFCDVRFSTCDNQKSCMSNCSITSICEKPQEVCVAVWRKNDENITLETVCHDPKLPYHDFILEDAASPKCIMKEKKKPGETFFMCSCSSDECNDNIIFSEEYNTSNPDLLLVIFQVTGISLLPPLGVAISVIIIFYCYRVNRQQKLSSTWETGKTRKLMEFSEHCAIILEDDRSDISSTCANNINHNTELLPIELDTLVGKGRFAEVYKAKLKQNTSEQFETVAVKIFPYEEYASWKTEKDIFSDINLKHENILQFLTAEERKTELGKQYWLITAFHAKGNLQEYLTRHVISWEDLRKLGSSLARGIAHLHSDHTPCGRPKMPIVHRDLKSSNILVKNDLTCCLCDFGLSLRLDPTLSVDDLANSGQVGTARYMAPEVLESRMNLENVESFKQTDVYSMALVLWEMTSRCNAVGEVKDYEPPFGSKVREHPCVESMKDNVLRDRGRPEIPSFWLNHQGIQMVCETLTECWDHDPEARLTAQCVAERFSELEHLDRLSGRSCSEEKIPEDGSLNTTK",
# TIE1_HUMAN
"MVWRVPPFLLPILFLASHVGAAVDLTLLANLRLTDPQRFFLTCVSGEAGAGRGSDAWGPPLLLEKDDRIVRTPPGPPLRLARNGSHQVTLRGFSKPSDLVGVFSCVGGAGARRTRVIYVHNSPGAHLLPDKVTHTVNKGDTAVLSARVHKEKQTDVIWKSNGSYFYTLDWHEAQDGRFLLQLPNVQPPSSGIYSATYLEASPLGSAFFRLIVRGCGAGRWGPGCTKECPGCLHGGVCHDHDGECVCPPGFTGTRCEQACREGRFGQSCQEQCPGISGCRGLTFCLPDPYGCSCGSGWRGSQCQEACAPGHFGADCRLQCQCQNGGTCDRFSGCVCPSGWHGVHCEKSDRIPQILNMASELEFNLETMPRINCAAAGNPFPVRGSIELRKPDGTVLLSTKAIVEPEKTTAEFEVPRLVLADSGFWECRVSTSGGQDSRRFKVNVKVPPVPLAAPRLLTKQSRQLVVSPLVSFSGDGPISTVRLHYRPQDSTMDWSTIVVDPSENVTLMNLRPKTGYSVRVQLSRPGEGGEGAWGPPTLMTTDCPEPLLQPWLEGWHVEGTDRLRVSWSLPLVPGPLVGDGFLLRLWDGTRGQERRENVSSPQARTALLTGLTPGTHYQLDVQLYHCTLLGPASPPAHVLLPPSGPPAPRHLHAQALSDSEIQLTWKHPEALPGPISKYVVEVQVAGGAGDPLWIDVDRPEETSTIIRGLNASTRYLFRMRASIQGLGDWSNTVEESTLGNGLQAEGPVQESRAAEEGLDQQLILAVVGSVSATCLTILAALLTLVCIRRSCLHRRRTFTYQSGSGEETILQFSSGTLTLTRRPKLQPEPLSYPVLEWEDITFEDLIGEGNFGQVIRAMIKKDGLKMNAAIKMLKEYASENDHRDFAGELEVLCKLGHHPNIINLLGACKNRGYLYIAIEYAPYGNLLDFLRKSRVLETDPAFAREHGTASTLSSRQLLRFASDAANGMQYLSEKQFIHRDLAARNVLVGENLASKIADFGLSRGEEVYVKKTMGRLPVRWMAIESLNYSVYTTKSDVWSFGVLLWEIVSLGGTPYCGMTCAELYEKLPQGYRMEQPRNCDDEVYELMRQCWRDRPYERPPFAQIALQLGRMLEARKAYVNMSLFENFTYAGIDATAEEA",
# TLK1_HUMAN
"MSVQSSSGSLEGPPSWSQLSTSPTPGSAAAARSLLNHTPPSGRPREGAMDELHSLDPRRQELLEARFTGVASGSTGSTGSCSVGAKASTNNESSNHSFGSLGSLSDKESETPEKKQSESSRGRKRKAENQNESSQGKSIGGRGHKISDYFEYQGGNGSSPVRGIPPAIRSPQNSHSHSTPSSSVRPNSPSPTALAFGDHPIVQPKQLSFKIIQTDLTMLKLAALESNKIQDLEKKEGRIDDLLRANCDLRRQIDEQQKLLEKYKERLNKCISMSKKLLIEKSTQEKLSSREKSMQDRLRLGHFTTVRHGASFTEQWTDGFAFQNLVKQQEWVNQQREDIERQRKLLAKRKPPTANNSQAPSTNSEPKQRKNKAVNGAENDPFVRPNLPQLLTLAEYHEQEEIFKLRLGHLKKEEAEIQAELERLERVRNLHIRELKRINNEDNSQFKDHPTLNERYLLLHLLGRGGFSEVYKAFDLYEQRYAAVKIHQLNKSWRDEKKENYHKHACREYRIHKELDHPRIVKLYDYFSLDTDTFCTVLEYCEGNDLDFYLKQHKLMSEKEARSIVMQIVNALRYLNEIKPPIIHYDLKPGNILLVDGTACGEIKITDFGLSKIMDDDSYGVDGMDLTSQGAGTYWYLPPECFVVGKEPPKISNKVDVWSVGVIFFQCLYGRKPFGHNQSQQDILQENTILKATEVQFPVKPVVSSEAKAFIRRCLAYRKEDRFDVHQLANDPYLLPHMRRSNSSGNLHMAGLTASPTPPSSSIITY",
# TLK2_HUMAN
"MMEELHSLDPRRQELLEARFTGVGVSKGPLNSESSNQSLCSVGSLSDKEVETPEKKQNDQRNRKRKAEPYETSQGKGTPRGHKISDYFEFAGGSAPGTSPGRSVPPVARSSPQHSLSNPLPRRVEQPLYGLDGSAAKEATEEQSALPTLMSVMLAKPRLDTEQLAQRGAGLCFTFVSAQQNSPSSTGSGNTEHSCSSQKQISIQHRQTQSDLTIEKISALENSKNSDLEKKEGRIDDLLRANCDLRRQIDEQQKMLEKYKERLNRCVTMSKKLLIEKSKQEKMACRDKSMQDRLRLGHFTTVRHGASFTEQWTDGYAFQNLIKQQERINSQREEIERQRKMLAKRKPPAMGQAPPATNEQKQRKSKTNGAENETPSSGNTELKDTAPALGAHSLLRLTLAEYHEQEEIFKLRLGHLKKEEAEIQAELERLERVRNLHIRELKRIHNEDNSQFKDHPTLNDRYLLLHLLGRGGFSEVYKAFDLTEQRYVAVKIHQLNKNWRDEKKENYHKHACREYRIHKELDHPRIVKLYDYFSLDTDSFCTVLEYCEGNDLDFYLKQHKLMSEKEARSIIMQIVNALKYLNEIKPPIIHYDLKPGNILLVNGTACGEIKITDFGLSKIMDDDSYNSVDGMELTSQGAGTYWYLPPECFVVGKEPPKISNKVDVWSVGVIFYQCLYGRKPFGHNQSQQDILQENTILKATEVQFPPKPVVTPEAKAFIRRCLAYRKEDRIDVQQLACDPYLLPHIRKSVSTSSPAGAAIASTSGASNNSSSN",
# TNK1_HUMAN
"MLPEAGSLWLLKLLRDIQLAQFYWPILEELNVTRPEHFDFVKPEDLDGIGMGRPAQRRLSEALKRLRSGPKSKNWVYKILGGFAPEHKEPTLPSDSPRHLPEPEGGLKCLIPEGAVCRGELLGSGCFGVVHRGLWTLPSGKSVPVAVKSLRVGPEGPMGTELGDFLREVSVMMNLEHPHVLRLHGLVLGQPLQMVMELAPLGSLHARLTAPAPTPPLLVALLCLFLRQLAGAMAYLGARGLVHRDLATRNLLLASPRTIKVADFGLVRPLGGARGRYVMGGPRPIPYAWCAPESLRHGAFSSASDVWMFGVTLWEMFSGGEEPWAGVPPYLILQRLEDRARLPRPPLCSRALYSLALRCWAPHPADRPSFSHLEGLLQEAGPSEACCVRDVTEPGALRMETGDPITVIEGSSSFHSPDSTIWKGQNGRTFKVGSFPASAVTLADAGGLPATRPVHRGTPARGDQHPGSIDGDRKKANLWDAPPARGQRRNMPLERMKGISRSLESVLSLGPRPTGGGSSPPEIRQARAVPQGPPGLPPRPPLSSSSPQPSQPSRERLPWPKRKPPHNHPMGMPGARKAAALSGGLLSDPELQRKIMEVELSVHGVTHQECQTALGATGGDVVSAIRNLKVDQLFHLSSRSRADCWRILEHYQWDLSAASRYVLARP",
# TRIO_HUMAN
"MSGSSGGAAAPAASSGPAAAASAAGSGCGGGAGEGAEEAAKDLADIAAFFRSGFRKNDEMKAMDVLPILKEKVAYLSGGRDKRGGPILTFPARSNHDRIRQEDLRRLISYLACIPSEEVCKRGFTVIVDMRGSKWDSIKPLLKILQESFPCCIHVALIIKPDNFWQKQRTNFGSSKFEFETNMVSLEGLTKVVDPSQLTPEFDGCLEYNHEEWIEIRVAFEDYISNATHMLSRLEELQDILAKKELPQDLEGARNMIEEHSQLKKKVIKAPIEDLDLEGQKLLQRIQSSESFPKKNSGSGNADLQNLLPKVSTMLDRLHSTRQHLHQMWHVRKLKLDQCFQLRLFEQDAEKMFDWITHNKGLFLNSYTEIGTSHPHAMELQTQHNHFAMNCMNVYVNINRIMSVANRLVESGHYASQQIRQIASQLEQEWKAFAAALDERSTLLDMSSIFHQKAEKYMSNVDSWCKACGEVDLPSELQDLEDAIHHHQGIYEHITLAYSEVSQDGKSLLDKLQRPLTPGSSDSLTASANYSKAVHHVLDVIHEVLHHQRQLENIWQHRKVRLHQRLQLCVFQQDVQQVLDWIENHGEAFLSKHTGVGKSLHRARALQKRHEDFEEVAQNTYTNADKLLEAAEQLAQTGECDPEEIYQAAHQLEDRIQDFVRRVEQRKILLDMSVSFHTHVKELWTWLEELQKELLDDVYAESVEAVQDLIKRFGQQQQTTLQVTVNVIKEGEDLIQQLRDSAISSNKTPHNSSINHIETVLQQLDEAQSQMEELFQERKIKLELFLQLRIFERDAIDIISDLESWNDELSQQMNDFDTEDLTIAEQRLQHHADKALTMNNLTFDVIHQGQDLLQYVNEVQASGVELLCDRDVDMATRVQDLLEFLHEKQQELDLAAEQHRKHLEQCVQLRHLQAEVKQVLGWIRNGESMLNAGLITASSLQEAEQLQREHEQFQHAIEKTHQSALQVQQKAEAMLQANHYDMDMIRDCAEKVASHWQQLMLKMEDRLKLVNASVAFYKTSEQVCSVLESLEQEYKREEDWCGGADKLGPNSETDHVTPMISKHLEQKEAFLKACTLARRNADVFLKYLHRNSVNMPGMVTHIKAPEQQVKNILNELFQRENRVLHYWTMRKRRLDQCQQYVVFERSAKQALEWIHDNGEFYLSTHTSTGSSIQHTQELLKEHEEFQITAKQTKERVKLLIQLADGFCEKGHAHAAEIKKCVTAVDKRYRDFSLRMEKYRTSLEKALGISSDSNKSSKSLQLDIIPASIPGSEVKLRDAAHELNEEKRKSARRKEFIMAELIQTEKAYVRDLRECMDTYLWEMTSGVEEIPPGIVNKELIIFGNMQEIYEFHNNIFLKELEKYEQLPEDVGHCFVTWADKFQMYVTYCKNKPDSTQLILEHAGSYFDEIQQRHGLANSISSYLIKPVQRITKYQLLLKELLTCCEEGKGEIKDGLEVMLSVPKRANDAMHLSMLEGFDENIESQGELILQESFQVWDPKTLIRKGRERHLFLFEMSLVFSKEVKDSSGRSKYLYKSKLFTSELGVTEHVEGDPCKFALWVGRTPTSDNKIVLKASSIENKQDWIKHIREVIQERTIHLKGALKEPIHIPKTAPATRQKGRRDGEDLDSQGDGSSQPDTISIASRTSQNTLDSDKLSGGCELTVVIHDFTACNSNELTIRRGQTVEVLERPHDKPDWCLVRTTDRSPAAEGLVPCGSLCIAHSRSSMEMEGIFNHKDSLSVSSNDASPPASVASLQPHMIGAQSSPGPKRPGNTLRKWLTSPVRRLSSGKADGHVKKLAHKHKKSREVRKSADAGSQKDSDDSAATPQDETVEERGRNEGLSSGTLSKSSSSGMQSCGEEEGEEGADAVPLPPPMAIQQHSLLQPDSQDDKASSRLLVRPTSSETPSAAELVSAIEELVKSKMALEDRPSSLLVDQGDSSSPSFNPSDNSLLSSSSPIDEMEERKSSSLKRRHYVLQELVETERDYVRDLGYVVEGYMALMKEDGVPDDMKGKDKIVFGNIHQIYDWHRDFFLGELEKCLEDPEKLGSLFVKHERRLHMYIAYCQNKPKSEHIVSEYIDTFFEDLKQRLGHRLQLTDLLIKPVQRIMKYQLLLKDFLKYSKKASLDTSELERAVEVMCIVPRRCNDMMNVGRLQGFDGKIVAQGKLLLQDTFLVTDQDAGLLPRCRERRIFLFEQIVIFSEPLDKKKGFSMPGFLFKNSIKVSCLCLEENVENDPCKFALTSRTGDVVETFILHSSSPSVRQTWIHEINQILENQRNFLNALTSPIEYQRNHSGGGGGGGSGGSGGGGGSGGGGAPSGGSGHSGGPSSCGGAPSTSRSRPSRIPQPVRHHPPVLVSSAASSQAEADKMSGTSTPGPSLPPPGAAPEAGPSAPSRRPPGADAEGSEREAEPIPKMKVLESPRKGAANASGSSPDAPAKDARASLGTLPLGKPRAGAASPLNSPLSSAVPSLGKEPFPPSSPLQKGGSFWSSIPASPASRPGSFTFPGDSDSLQRQTPRHAAPGKDTDRMSTCSSASEQSVQSTQSNGSESSSSSNISTMLVTHDYTAVKEDEINVYQGEVVQILASNQQNMFLVFRAATDQCPAAEGWIPGFVLGHTSAVIVENPDGTLKKSTSWHTALRLRKKSEKKDKDGKREGKLENGYRKSREGLSNKVSVKLLNPNYIYDVPPEFVIPLSEVTCETGETVVLRCRVCGRPKASITWKGPEHNTLNNDGHYSISYSDLGEATLKIVGVTTEDDGIYTCIAVNDMGSASSSASLRVLGPGMDGIMVTWKDNFDSFYSEVAELGRGRFSVVKKCDQKGTKRAVATKFVNKKLMKRDQVTHELGILQSLQHPLLVGLLDTFETPTSYILVLEMADQGRLLDCVVRWGSLTEGKIRAHLGEVLEAVRYLHNCRIAHLDLKPENILVDESLAKPTIKLADFGDAVQLNTTYYIHQLLGNPEFAAPEIILGNPVSLTSDTWSVGVLTYVLLSGVSPFLDDSVEETCLNICRLDFSFPDDYFKGVSQKAKEFVCFLLQEDPAKRPSAALALQEQWLQAGNGRSTGVLDTSRLTSFIERRKHQNDVRPIRSIKNFLQSRLLPRV",
# NTRK1_HUMAN
"MLRGGRRGQLGWHSWAAGPGSLLAWLILASAGAAPCPDACCPHGSSGLRCTRDGALDSLHHLPGAENLTELYIENQQHLQHLELRDLRGLGELRNLTIVKSGLRFVAPDAFHFTPRLSRLNLSFNALESLSWKTVQGLSLQELVLSGNPLHCSCALRWLQRWEEEGLGGVPEQKLQCHGQGPLAHMPNASCGVPTLKVQVPNASVDVGDDVLLRCQVEGRGLEQAGWILTELEQSATVMKSGGLPSLGLTLANVTSDLNRKNVTCWAENDVGRAEVSVQVNVSFPASVQLHTAVEMHHWCIPFSVDGQPAPSLRWLFNGSVLNETSFIFTEFLEPAANETVRHGCLRLNQPTHVNNGNYTLLAANPFGQASASIMAAFMDNPFEFNPEDPIPVSFSPVDTNSTSGDPVEKKDETPFGVSVAVGLAVFACLFLSTLLLVLNKCGRRNKFGINRPAVLAPEDGLAMSLHFMTLGGSSLSPTEGKGSGLQGHIIENPQYFSDACVHHIKRRDIVLKWELGEGAFGKVFLAECHNLLPEQDKMLVAVKALKEASESARQDFQREAELLTMLQHQHIVRFFGVCTEGRPLLMVFEYMRHGDLNRFLRSHGPDAKLLAGGEDVAPGPLGLGQLLAVASQVAAGMVYLAGLHFVHRDLATRNCLVGQGLVVKIGDFGMSRDIYSTDYYRVGGRTMLPIRWMPPESILYRKFTTESDVWSFGVVLWEIFTYGKQPWYQLSNTEAIDCITQGRELERPRACPPEVYAIMRGCWQREPQQRHSIKDVHARLQALAQAPPVYLDVLG",
# NTRK2_HUMAN
"MSSWIRWHGPAMARLWGFCWLVVGFWRAAFACPTSCKCSASRIWCSDPSPGIVAFPRLEPNSVDPENITEIFIANQKRLEIINEDDVEAYVGLRNLTIVDSGLKFVAHKAFLKNSNLQHINFTRNKLTSLSRKHFRHLDLSELILVGNPFTCSCDIMWIKTLQEAKSSPDTQDLYCLNESSKNIPLANLQIPNCGLPSANLAAPNLTVEEGKSITLSCSVAGDPVPNMYWDVGNLVSKHMNETSHTQGSLRITNISSDDSGKQISCVAENLVGEDQDSVNLTVHFAPTITFLESPTSDHHWCIPFTVKGNPKPALQWFYNGAILNESKYICTKIHVTNHTEYHGCLQLDNPTHMNNGDYTLIAKNEYGKDEKQISAHFMGWPGIDDGANPNYPDVIYEDYGTAANDIGDTTNRSNEIPSTDVTDKTGREHLSVYAVVVIASVVGFCLLVMLFLLKLARHSKFGMKGPASVISNDDDSASPLHHISNGSNTPSSSEGGPDAVIIGMTKIPVIENPQYFGITNSQLKPDTFVQHIKRHNIVLKRELGEGAFGKVFLAECYNLCPEQDKILVAVKTLKDASDNARKDFHREAELLTNLQHEHIVKFYGVCVEGDPLIMVFEYMKHGDLNKFLRAHGPDAVLMAEGNPPTELTQSQMLHIAQQIAAGMVYLASQHFVHRDLATRNCLVGENLLVKIGDFGMSRDVYSTDYYRVGGHTMLPIRWMPPESIMYRKFTTESDVWSLGVVLWEIFTYGKQPWYQLSNNEVIECITQGRVLQRPRTCPQEVYELMLGCWQREPHMRKNIKGIHTLLQNLAKASPVYLDILG",
# NTRK3_HUMAN
"MDVSLCPAKCSFWRIFLLGSVWLDYVGSVLACPANCVCSKTEINCRRPDDGNLFPLLEGQDSGNSNGNASINITDISRNITSIHIENWRSLHTLNAVDMELYTGLQKLTIKNSGLRSIQPRAFAKNPHLRYINLSSNRLTTLSWQLFQTLSLRELQLEQNFFNCSCDIRWMQLWQEQGEAKLNSQNLYCINADGSQLPLFRMNISQCDLPEISVSHVNLTVREGDNAVITCNGSGSPLPDVDWIVTGLQSINTHQTNLNWTNVHAINLTLVNVTSEDNGFTLTCIAENVVGMSNASVALTVYYPPRVVSLEEPELRLEHCIEFVVRGNPPPTLHWLHNGQPLRESKIIHVEYYQEGEISEGCLLFNKPTHYNNGNYTLIAKNPLGTANQTINGHFLKEPFPESTDNFILFDEVSPTPPITVTHKPEEDTFGVSIAVGLAAFACVLLVVLFVMINKYGRRSKFGMKGPVAVISGEEDSASPLHHINHGITTPSSLDAGPDTVVIGMTRIPVIENPQYFRQGHNCHKPDTYVQHIKRRDIVLKRELGEGAFGKVFLAECYNLSPTKDKMLVAVKALKDPTLAARKDFQREAELLTNLQHEHIVKFYGVCGDGDPLIMVFEYMKHGDLNKFLRAHGPDAMILVDGQPRQAKGELGLSQMLHIASQIASGMVYLASQHFVHRDLATRNCLVGANLLVKIGDFGMSRDVYSTDYYRLFNPSGNDFCIWCEVGGHTMLPIRWMPPESIMYRKFTTESDVWSFGVILWEIFTYGKQPWFQLSNTEVIECITQGRVLERPRVCPKEVYDVMLGCWQREPQQRLNIKEIYKILHALGKATPIYLDILG",
# TRRAP_HUMAN
"MAFVATQGATVVDQTTLMKKYLQFVAALTDVNTPDETKLKMMQEVSENFENVTSSPQYSTFLEHIIPRFLTFLQDGEVQFLQEKPAQQLRKLVLEIIHRIPTNEHLRPHTKNVLSVMFRFLETENEENVLICLRIIIELHKQFRPPITQEIHHFLDFVKQIYKELPKVVNRYFENPQVIPENTVPPPEMVGMITTIAVKVNPEREDSETRTHSIIPRGSLSLKVLAELPIIVVLMYQLYKLNIHNVVAEFVPLIMNTIAIQVSAQARQHKLYNKELYADFIAAQIKTLSFLAYIIRIYQELVTKYSQQMVKGMLQLLSNCPAETAHLRKELLIAAKHILTTELRNQFIPCMDKLFDESILIGSGYTARETLRPLAYSTLADLVHHVRQHLPLSDLSLAVQLFAKNIDDESLPSSIQTMSCKLLLNLVDCIRSKSEQESGNGRDVLMRMLEVFVLKFHTIARYQLSAIFKKCKPQSELGAVEAALPGVPTAPAAPGPAPSPAPVPAPPPPPPPPPPATPVTPAPVPPFEKQGEKDKEDKQTFQVTDCRSLVKTLVCGVKTITWGITSCKAPGEAQFIPNKQLQPKETQIYIKLVKYAMQALDIYQVQIAGNGQTYIRVANCQTVRMKEEKEVLEHFAGVFTMMNPLTFKEIFQTTVPYMVERISKNYALQIVANSFLANPTTSALFATILVEYLLDRLPEMGSNVELSNLYLKLFKLVFGSVSLFAAENEQMLKPHLHKIVNSSMELAQTAKEPYNYFLLLRALFRSIGGGSHDLLYQEFLPLLPNLLQGLNMLQSGLHKQHMKDLFVELCLTVPVRLSSLLPYLPMLMDPLVSALNGSQTLVSQGLRTLELCVDNLQPDFLYDHIQPVRAELMQALWRTLRNPADSISHVAYRVLGKFGGSNRKMLKESQKLHYVVTEVQGPSITVEFSDCKASLQLPMEKAIETALDCLKSANTEPYYRRQAWEVIKCFLVAMMSLEDNKHALYQLLAHPNFTEKTIPNVIISHRYKAQDTPARKTFEQALTGAFMSAVIKDLRPSALPFVASLIRHYTMVAVAQQCGPFLLPCYQVGSQPSTAMFHSEENGSKGMDPLVLIDAIAICMAYEEKELCKIGEVALAVIFDVASIILGSKERACQLPLFSYIVERLCACCYEQAWYAKLGGVVSIKFLMERLPLTWVLQNQQTFLKALLFVMMDLTGEVSNGAVAMAKTTLEQLLMRCATPLKDEERAEEIVAAQEKSFHHVTHDLVREVTSPNSTVRKQAMHSLQVLAQVTGKSVTVIMEPHKEVLQDMVPPKKHLLRHQPANAQIGLMEGNTFCTTLQPRLFTMDLNVVEHKVFYTELLNLCEAEDSALTKLPCYKSLPSLVPLRIAALNALAACNYLPQSREKIIAALFKALNSTNSELQEAGEACMRKFLEGATIEVDQIHTHMRPLLMMLGDYRSLTLNVVNRLTSVTRLFPNSFNDKFCDQMMQHLRKWMEVVVITHKGGQRSDGNESISECGRCPLSPFCQFEEMKICSAIINLFHLIPAAPQTLVKPLLEVVMKTERAMLIEAGSPFREPLIKFLTRHPSQTVELFMMEATLNDPQWSRMFMSFLKHKDARPLRDVLAANPNRFITLLLPGGAQTAVRPGSPSTSTMRLDLQFQAIKIISIIVKNDDSWLASQHSLVSQLRRVWVSENFQERHRKENMAATNWKEPKLLAYCLLNYCKRNYGDIELLFQLLRAFTGRFLCNMTFLKEYMEEEIPKNYSIAQKRALFFRFVDFNDPNFGDELKAKVLQHILNPAFLYSFEKGEGEQLLGPPNPEGDNPESITSVFITKVLDPEKQADMLDSLRIYLLQYATLLVEHAPHHIHDNNKNRNSKLRRLMTFAWPCLLSKACVDPACKYSGHLLLAHIIAKFAIHKKIVLQVFHSLLKAHAMEARAIVRQAMAILTPAVPARMEDGHQMLTHWTRKIIVEEGHTVPQLVHILHLIVQHFKVYYPVRHHLVQHMVSAMQRLGFTPSVTIEQRRLAVDLSEVVIKWELQRIKDQQPDSDMDPNSSGEGVNSVSSSIKRGLSVDSAQEVKRFRTATGAISAVFGRSQSLPGADSLLAKPIDKQHTDTVVNFLIRVACQVNDNTNTAGSPGEVLSRRCVNLLKTALRPDMWPKSELKLQWFDKLLMTVEQPNQVNYGNICTGLEVLSFLLTVLQSPAILSSFKPLQRGIAACMTCGNTKVLRAVHSLLSRLMSIFPTEPSTSSVASKYEELECLYAAVGKVIYEGLTNYEKATNANPSQLFGTLMILKSACSNNPSYIDRLISVFMRSLQKMVREHLNPQAASGSTEATSGTSELVMLSLELVKTRLAVMSMEMRKNFIQAILTSLIEKSPDAKILRAVVKIVEEWVKNNSPMAANQTPTLREKSILLVKMMTYIEKRFPEDLELNAQFLDLVNYVYRDETLSGSELTAKLEPAFLSGLRCAQPLIRAKFFEVFDNSMKRRVYERLLYVTCSQNWEAMGNHFWIKQCIELLLAVCEKSTPIGTSCQGAMLPSITNVINLADSHDRAAFAMVTHVKQEPRERENSESKEEDVEIDIELAPGDQTSTPKTKELSEKDIGNQLHMLTNRHDKFLDTLREVKTGALLSAFVQLCHISTTLAEKTWVQLFPRLWKILSDRQQHALAGEISPFLCSGSHQVQRDCQPSALNCFVEAMSQCVPPIPIRPCVLKYLGKTHNLWFRSTLMLEHQAFEKGLSLQIKPKQTTEFYEQESITPPQQEILDSLAELYSLLQEEDMWAGLWQKRCKYSETATAIAYEQHGFFEQAQESYEKAMDKAKKEHERSNASPAIFPEYQLWEDHWIRCSKELNQWEALTEYGQSKGHINPYLVLECAWRVSNWTAMKEALVQVEVSCPKEMAWKVNMYRGYLAICHPEEQQLSFIERLVEMASSLAIREWRRLPHVVSHVHTPLLQAAQQIIELQEAAQINAGLQPTNLGRNNSLHDMKTVVKTWRNRLPIVSDDLSHWSSIFMWRQHHYQGKPTWSGMHSSSIVTAYENSSQHDPSSNNAMLGVHASASAIIQYGKIARKQGLVNVALDILSRIHTIPTVPIVDCFQKIRQQVKCYLQLAGVMGKNECMQGLEVIESTNLKYFTKEMTAEFYALKGMFLAQINKSEEANKAFSAAVQMHDVLVKAWAMWGDYLENIFVKERQLHLGVSAITCYLHACRHQNESKSRKYLAKVLWLLSFDDDKNTLADAVDKYCIGVPPIQWLAWIPQLLTCLVGSEGKLLLNLISQVGRVYPQAVYFPIRTLYLTLKIEQRERYKSDPGPIRATAPMWRCSRIMHMQRELHPTLLSSLEGIVDQMVWFRENWHEEVLRQLQQGLAKCYSVAFEKSGAVSDAKITPHTLNFVKKLVSTFGVGLENVSNVSTMFSSAASESLARRAQATAQDPVFQKLKGQFTTDFDFSVPGSMKLHNLISKLKKWIKILEAKTKQLPKFFLIEEKCRFLSNFSAQTAEVEIPGEFLMPKPTHYYIKIARFMPRVEIVQKHNTAARRLYIRGHNGKIYPYLVMNDACLTESRREERVLQLLRLLNPCLEKRKETTKRHLFFTVPRVVAVSPQMRLVEDNPSSLSLVEIYKQRCAKKGIEHDNPISRYYDRLATVQARGTQASHQVLRDILKEVQSNMVPRSMLKEWALHTFPNATDYWTFRKMFTIQLALIGFAEFVLHLNRLNPEMLQIAQDTGKLNVAYFRFDINDATGDLDANRPVPFRLTPNISEFLTTIGVSGPLTASMIAVARCFAQPNFKVDGILKTVLRDEIIAWHKKTQEDTSSPLSAAGQPENMDSQQLVSLVQKAVTAIMTRLHNLAQFEGGESKVNTLVAAANSLDNLCRMDPAWHPWL",
# TTK_HUMAN
"MESEDLSGRELTIDSIMNKVRDIKNKFKNEDLTDELSLNKISADTTDNSGTVNQIMMMANNPEDWLSLLLKLEKNSVPLSDALLNKLIGRYSQAIEALPPDKYGQNESFARIQVRFAELKAIQEPDDARDYFQMARANCKKFAFVHISFAQFELSQGNVKKSKQLLQKAVERGAVPLEMLEIALRNLNLQKKQLLSEEEKKNLSASTVLTAQESFSGSLGHLQNRNNSCDSRGQTTKARFLYGENMPPQDAEIGYRNSLRQTNKTKQSCPFGRVPVNLLNSPDCDVKTDDSVVPCFMKRQTSRSECRDLVVPGSKPSGNDSCELRNLKSVQNSHFKEPLVSDEKSSELIITDSITLKNKTESSLLAKLEETKEYQEPEVPESNQKQWQSKRKSECINQNPAASSNHWQIPELARKVNTEQKHTTFEQPVFSVSKQSPPISTSKWFDPKSICKTPSSNTLDDYMSCFRTPVVKNDFPPACQLSTPYGQPACFQQQQHQILATPLQNLQVLASSSANECISVKGRIYSILKQIGSGGSSKVFQVLNEKKQIYAIKYVNLEEADNQTLDSYRNEIAYLNKLQQHSDKIIRLYDYEITDQYIYMVMECGNIDLNSWLKKKKSIDPWERKSYWKNMLEAVHTIHQHGIVHSDLKPANFLIVDGMLKLIDFGIANQMQPDTTSVVKDSQVGTVNYMPPEAIKDMSSSRENGKSKSKISPKSDVWSLGCILYYMTYGKTPFQQIINQISKLHAIIDPNHEIEFPDIPEKDLQDVLKCCLKRDPKQRISIPELLAHPYVQIQTHPVNQMAKGTTEEMKYVLGQLVGLNSPNSILKAAKTLYEHYSGGESHNSSSSKTFEKKRGKK",
# TXK_HUMAN
"MILSSYNTIQSVFCCCCCCSVQKRQMRTQISLSTDEELPEKYTQRRRPWLSQLSNKKQSNTGRVQPSKRKPLPPLPPSEVAEEKIQVKALYDFLPREPCNLALRRAEEYLILEKYNPHWWKARDRLGNEGLIPSNYVTENKITNLEIYEWYHRNITRNQAEHLLRQESKEGAFIVRDSRHLGSYTISVFMGARRSTEAAIKHYQIKKNDSGQWYVAERHAFQSIPELIWYHQHNAAGLMTRLRYPVGLMGSCLPATAGFSYEKWEIDPSELAFIKEIGSGQFGVVHLGEWRSHIQVAIKAINEGSMSEEDFIEEAKVMMKLSHSKLVQLYGVCIQRKPLYIVTEFMENGCLLNYLRENKGKLRKEMLLSVCQDICEGMEYLERNGYIHRDLAARNCLVSSTCIVKISDFGMTRYVLDDEYVSSFGAKFPIKWSPPEVFLFNKYSSKSDVWSFGVLMWEVFTEGKMPFENKSNLQVVEAISEGFRLYRPHLAPMSIYEVMYSCWHEKPEGRPTFAELLRAVTEIAETW",
# TYK2_HUMAN
"MPLRHWGMARGSKPVGDGAQPMAAMGGLKVLLHWAGPGGGEPWVTFSESSLTAEEVCIHIAHKVGITPPCFNLFALFDAQAQVWLPPNHILEIPRDASLMLYFRIRFYFRNWHGMNPREPAVYRCGPPGTEASSDQTAQGMQLLDPASFEYLFEQGKHEFVNDVASLWELSTEEEIHHFKNESLGMAFLHLCHLALRHGIPLEEVAKKTSFKDCIPRSFRRHIRQHSALTRLRLRNVFRRFLRDFQPGRLSQQMVMVKYLATLERLAPRFGTERVPVCHLRLLAQAEGEPCYIRDSGVAPTDPGPESAAGPPTHEVLVTGTGGIQWWPVEEEVNKEEGSSGSSGRNPQASLFGKKAKAHKAVGQPADRPREPLWAYFCDFRDITHVVLKEHCVSIHRQDNKCLELSLPSRAAALSFVSLVDGYFRLTADSSHYLCHEVAPPRLVMSIRDGIHGPLLEPFVQAKLRPEDGLYLIHWSTSHPYRLILTVAQRSQAPDGMQSLRLRKFPIEQQDGAFVLEGWGRSFPSVRELGAALQGCLLRAGDDCFSLRRCCLPQPGETSNLIIMRGARASPRTLNLSQLSFHRVDQKEITQLSHLGQGTRTNVYEGRLRVEGSGDPEEGKMDDEDPLVPGRDRGQELRVVLKVLDPSHHDIALAFYETASLMSQVSHTHLAFVHGVCVRGPENIMVTEYVEHGPLDVWLRRERGHVPMAWKMVVAQQLASALSYLENKNLVHGNVCGRNILLARLGLAEGTSPFIKLSDPGVGLGALSREERVERIPWLAPECLPGGANSLSTAMDKWGFGATLLEICFDGEAPLQSRSPSEKEHFYQRQHRLPEPSCPQLATLTSQCLTYEPTQRPSFRTILRDLTRLQPHNLADVLTVNPDSPASDPTVFHKRYLKKIRDLGEGHFGKVSLYCYDPTNDGTGEMVAVKALKADCGPQHRSGWKQEIDILRTLYHEHIIKYKGCCEDQGEKSLQLVMEYVPLGSLRDYLPRHSIGLAQLLLFAQQICEGMAYLHAQHYIHRDLAARNVLLDNDRLVKIGDFGLAKAVPEGHEYYRVREDGDSPVFWYAPECLKEYKFYYASDVWSFGVTLYELLTHCDSSQSPPTKFLELIGIAQGQMTVLRLTELLERGERLPRPDKCPCEVYHLMKNCWETEASFRPTFENLIPILKTVHEKYQGQAPSVFSVC",
# TYRO3_HUMAN
"MALRRSMGRPGLPPLPLPPPPRLGLLLAALASLLLPESAAAGLKLMGAPVKLTVSQGQPVKLNCSVEGMEEPDIQWVKDGAVVQNLDQLYIPVSEQHWIGFLSLKSVERSDAGRYWCQVEDGGETEISQPVWLTVEGVPFFTVEPKDLAVPPNAPFQLSCEAVGPPEPVTIVWWRGTTKIGGPAPSPSVLNVTGVTQSTMFSCEAHNLKGLASSRTATVHLQALPAAPFNITVTKLSSSNASVAWMPGADGRALLQSCTVQVTQAPGGWEVLAVVVPVPPFTCLLRDLVPATNYSLRVRCANALGPSPYADWVPFQTKGLAPASAPQNLHAIRTDSGLILEWEEVIPEAPLEGPLGPYKLSWVQDNGTQDELTVEGTRANLTGWDPQKDLIVRVCVSNAVGCGPWSQPLVVSSHDRAGQQGPPHSRTSWVPVVLGVLTALVTAAALALILLRKRRKETRFGQAFDSVMARGEPAVHFRAARSFNRERPERIEATLDSLGISDELKEKLEDVLIPEQQFTLGRMLGKGEFGSVREAQLKQEDGSFVKVAVKMLKADIIASSDIEEFLREAACMKEFDHPHVAKLVGVSLRSRAKGRLPIPMVILPFMKHGDLHAFLLASRIGENPFNLPLQTLIRFMVDIACGMEYLSSRNFIHRDLAARNCMLAEDMTVCVADFGLSRKIYSGDYYRQGCASKLPVKWLALESLADNLYTVQSDVWAFGVTMWEIMTRGQTPYAGIENAEIYNYLIGGNRLKQPPECMEDVYDLMYQCWSADPKQRPSFTCLRMELENILGQLSVLSASQDPLYINIERAEEPTAGGSLELPGRDQPYSGAGDGSGMGAVGGTPSDCRYILTPGGLAEQPGQAEHQPESPLNETQRLLLLQQGLLPHSSC",
# ULK1_HUMAN
"MEPGRGGTETVGKFEFSRKDLIGHGAFAVVFKGRHREKHDLEVAVKCINKKNLAKSQTLLGKEIKILKELKHENIVALYDFQEMANSVYLVMEYCNGGDLADYLHAMRTLSEDTIRLFLQQIAGAMRLLHSKGIIHRDLKPQNILLSNPAGRRANPNSIRVKIADFGFARYLQSNMMAATLCGSPMYMAPEVIMSQHYDGKADLWSIGTIVYQCLTGKAPFQASSPQDLRLFYEKNKTLVPTIPRETSAPLRQLLLALLQRNHKDRMDFDEFFHHPFLDASPSVRKSPPVPVPSYPSSGSGSSSSSSSTSHLASPPSLGEMQQLQKTLASPADTAGFLHSSRDSGGSKDSSCDTDDFVMVPAQFPGDLVAEAPSAKPPPDSLMCSGSSLVASAGLESHGRTPSPSPPCSSSPSPSGRAGPFSSSRCGASVPIPVPTQVQNYQRIERNLQSPTQFQTPRSSAIRRSGSTSPLGFARASPSPPAHAEHGGVLARKMSLGGGRPYTPSPQVGTIPERPGWSGTPSPQGAEMRGGRSPRPGSSAPEHSPRTSGLGCRLHSAPNLSDLHVVRPKLPKPPTDPLGAVFSPPQASPPQPSHGLQSCRNLRGSPKLPDFLQRNPLPPILGSPTKAVPSFDFPKTPSSQNLLALLARQGVVMTPPRNRTLPDLSEVGPFHGQPLGPGLRPGEDPKGPFGRSFSTSRLTDLLLKAAFGTQAPDPGSTESLQEKPMEIAPSAGFGGSLHPGARAGGTSSPSPVVFTVGSPPSGSTPPQGPRTRMFSAGPTGSASSSARHLVPGPCSEAPAPELPAPGHGCSFADPITANLEGAVTFEAPDLPEETLMEQEHTEILRGLRFTLLFVQHVLEIAALKGSASEAAGGPEYQLQESVVADQISLLSREWGFAEQLVLYLKVAELLSSGLQSAIDQIRAGKLCLSSTVKQVVRRLNELYKASVVSCQGLSLRLQRFFLDKQRLLDRIHSITAERLIFSHAVQMVQSAALDEMFQHREGCVPRYHKALLLLEGLQHMLSDQADIENVTKCKLCIERRLSALLTGICA",
# ULK2_HUMAN
"MEVVGDFEYSKRDLVGHGAFAVVFRGRHRQKTDWEVAIKSINKKNLSKSQILLGKEIKILKELQHENIVALYDVQELPNSVFLVMEYCNGGDLADYLQAKGTLSEDTIRVFLHQIAAAMRILHSKGIIHRDLKPQNILLSYANRRKSSVSGIRIKIADFGFARYLHSNMMAATLCGSPMYMAPEVIMSQHYDAKADLWSIGTVIYQCLVGKPPFQANSPQDLRMFYEKNRSLMPSIPRETSPYLANLLLGLLQRNQKDRMDFEAFFSHPFLEQGPVKKSCPVPVPMYSGSVSGSSCGSSPSCRFASPPSLPDMQHIQEENLSSPPLGPPNYLQVSKDSASTSSKNSSCDTDDFVLVPHNISSDHSCDMPVGTAGRRASNEFLVCGGQCQPTVSPHSETAPIPVPTQIRNYQRIEQNLTSTASSGTNVHGSPRSAVVRRSNTSPMGFLRPGSCSPVPADTAQTVGRRLSTGSSRPYSPSPLVGTIPEQFSQCCCGHPQGHDSRSRNSSGSPVPQAQSPQSLLSGARLQSAPTLTDIYQNKQKLRKQHSDPVCPSHTGAGYSYSPQPSRPGSLGTSPTKHLGSSPRSSDWFFKTPLPTIIGSPTKTTAPFKIPKTQASSNLLALVTRHGPAEEQSKDGNEPRECAHCLLVQGSERQRAEQQSKAVFGRSVSTGKLSDQQGKTPICRHQGSTDSLNTERPMDIAPAGACGGVLAPPAGTAASSKAVLFTVGSPPHSAAAPTCTHMFLRTRTTSVGPSNSGGSLCAMSGRVCVGSPPGPGFGSSPPGAEAAPSLRYVPYGASPPSLEGLITFEAPELPEETLMEREHTDTLRHLNVMLMFTECVLDLTAMRGGNPELCTSAVSLYQIQESVVVDQISQLSKDWGRVEQLVLYMKAAQLLAASLHLAKAQIKSGKLSPSTAVKQVVKNLNERYKFCITMCKKLTEKLNRFFSDKQRFIDEINSVTAEKLIYNCAVEMVQSAALDEMFQQTEDIVYRYHKAALLLEGLSRILQDPADIENVHKYKCSIERRLSALCHSTATV",
# VRK1_HUMAN
"MPRVKAAQAGRQSSAKRHLAEQFAVGEIITDMAKKEWKVGLPIGQGGFGCIYLADMNSSESVGSDAPCVVKVEPSDNGPLFTELKFYQRAAKPEQIQKWIRTRKLKYLGVPKYWGSGLHDKNGKSYRFMIMDRFGSDLQKIYEANAKRFSRKTVLQLSLRILDILEYIHEHEYVHGDIKASNLLLNYKNPDQVYLVDYGLAYRYCPEGVHKEYKEDPKRCHDGTIEFTSIDAHNGVAPSRRGDLEILGYCMIQWLTGHLPWEDNLKDPKYVRDSKIRYRENIASLMDKCFPEKNKPGEIAKYMETVKLLDYTEKPLYENLRDILLQGLKAIGSKDDGKLDLSVVENGGLKAKTITKKRKKEIEESKEPGVEDTEWSNTQTEEAIQTRSRTRKRVQK",
# VRK2_HUMAN
"MPPKRNEKYKLPIPFPEGKVLDDMEGNQWVLGKKIGSGGFGLIYLAFPTNKPEKDARHVVKVEYQENGPLFSELKFYQRVAKKDCIKKWIERKQLDYLGIPLFYGSGLTEFKGRSYRFMVMERLGIDLQKISGQNGTFKKSTVLQLGIRMLDVLEYIHENEYVHGDIKAANLLLGYKNPDQVYLADYGLSYRYCPNGNHKQYQENPRKGHNGTIEFTSLDAHKGVALSRRSDVEILGYCMLRWLCGKLPWEQNLKDPVAVQTAKTNLLDELPQSVLKWAPSGSSCCEIAQFLVCAHSLAYDEKPNYQALKKILNPHGIPLGPLDFSTKGQSINVHTPNSQKVDSQKAATKQVNKAHNRLIEKKVHSERSAESCATWKVQKEEKLIGLMNNEAAQESTRRRQKYQESQEPLNEVNSFPQKISYTQFPNSFYEPHQDFTSPDIFKKSRSPSWYKYTSTVSTGITDLESSTGLWPTISQFTLSEETNADVYYYRIIIPVLLMLVFLALFFL",
# WEE1_HUMAN
"MSFLSRQQPPPPRRAGAACTLRQKLIFSPCSDCEEEEEEEEEEGSGHSTGEDSAFQEPDSPLPPARSPTEPGPERRRSPGPAPGSPGELEEDLLLPGACPGADEAGGGAEGDSWEEEGFGSSSPVKSPAAPYFLGSSFSPVRCGGPGDASPRGCGARRAGEGRRSPRPDHPGTPPHKTFRKLRLFDTPHTPKSLLSKARGIDSSSVKLRGSSLFMDTEKSGKREFDVRQTPQVNINPFTPDSLLLHSSGQCRRRKRTYWNDSCGEDMEASDYELEDETRPAKRITITESNMKSRYTTEFHELEKIGSGEFGSVFKCVKRLDGCIYAIKRSKKPLAGSVDEQNALREVYAHAVLGQHSHVVRYFSAWAEDDHMLIQNEYCNGGSLADAISENYRIMSYFKEAELKDLLLQVGRGLRYIHSMSLVHMDIKPSNIFISRTSIPNAASEEGDEDDWASNKVMFKIGDLGHVTRISSPQVEEGDSRFLANEVLQENYTHLPKADIFALALTVVCAAGAEPLPRNGDQWHEIRQGRLPRIPQVLSQEFTELLKVMIHPDPERRPSAMALVKHSVLLSASRKSAEQLRIELNAEKFKNSLLQKELKKAQMAKAAAEERALFTDRMATRSTTQSNRTSRLIGKKMNRSVSLTIY",
# YES_HUMAN
"MGCIKSKENKSPAIKYRPENTPEPVSTSVSHYGAEPTTVSPCPSSSAKGTAVNFSSLSMTPFGGSSGVTPFGGASSSFSVVPSSYPAGLTGGVTIFVALYDYEARTTEDLSFKKGERFQIINNTEGDWWEARSIATGKNGYIPSNYVAPADSIQAEEWYFGKMGRKDAERLLLNPGNQRGIFLVRESETTKGAYSLSIRDWDEIRGDNVKHYKIRKLDNGGYYITTRAQFDTLQKLVKHYTEHADGLCHKLTTVCPTVKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTKVAIKTLKPGTMMPEAFLQEAQIMKKLRHDKLVPLYAVVSEEPIYIVTEFMSKGSLLDFLKEGDGKYLKLPQLVDMAAQIADGMAYIERMNYIHRDLRAANILVGENLVCKIADFGLARLIEDNEYTARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILQTELVTKGRVPYPGMVNREVLEQVERGYRMPCPQGCPESLHELMNLCWKKDPDERPTFEYIQSFLEDYFTATEPQYQPGENL",
# STK25_HUMAN
"MAHLRGFANQHSRVDPEELFTKLDRIGKGSFGEVYKGIDNHTKEVVAIKIIDLEEAEDEIEDIQQEITVLSQCDSPYITRYFGSYLKSTKLWIIMEYLGGGSALDLLKPGPLEETYIATILREILKGLDYLHSERKIHRDIKAANVLLSEQGDVKLADFGVAGQLTDTQIKRNTFVGTPFWMAPEVIKQSAYDFKADIWSLGITAIELAKGEPPNSDLHPMRVLFLIPKNSPPTLEGQHSKPFKEFVEACLNKDPRFRPTAKELLKHKFITRYTKKTSFLTELIDRYKRWKSEGHGEESSSEDSDIDGEAEDGEQGPIWTFPPTIRPSPHSKLHKGTALHSSQKPAEPVKRQPRSQCLSTLVRPVFGELKEKHKQSGGSVGALEELENAFSLAEESCPGISDKLMVHLVERVQRFSHNRNHLTSTR",
# ZAP70_HUMAN
"MPDPAAHLPFFYGSISRAEAEEHLKLAGMADGLFLLRQCLRSLGGYVLSLVHDVRFHHFPIERQLNGTYAIAGGKAHCGPAELCEFYSRDPDGLPCNLRKPCNRPSGLEPQPGVFDCLRDAMVRDYVRQTWKLEGEALEQAIISQAPQVEKLIATTAHERMPWYHSSLTREEAERKLYSGAQTDGKFLLRPRKEQGTYALSLIYGKTVYHYLISQDKAGKYCIPEGTKFDTLWQLVEYLKLKADGLIYCLKEACPNSSASNASGAAAPTLPAHPSTLTHPQRRIDTLNSDGYTPEPARITSPDKPRPMPMDTSVYESPYSDPEELKDKKLFLKRDNLLIADIELGCGNFGSVRQGVYRMRKKQIDVAIKVLKQGTEKADTEEMMREAQIMHQLDNPYIVRLIGVCQAEALMLVMEMAGGGPLHKFLVGKREEIPVSNVAELLHQVSMGMKYLEEKNFVHRDLAARNVLLVNRHYAKISDFGLSKALGADDSYYTARSAGKWPLKWYAPECINFRKFSSRSDVWSYGVTMWEALSYGQKPYKKMKGPEVMAFIEQGKRMECPPECPPELYALMSDCWIYKWEDRPDFLTVEQRMRACYYSLASKVEGPPGSTQKAEAACA",
# M3K13_HUMAN
"MANFQEHLSCSSSPHLPFSESKTFNGLQDELTAMGNHPSPKLLEDQQEKGMVRTELIESVHSPVTTTVLTSVSEDSRDQFENSVLQLREHDESETAVSQGNSNTVDGESTSGTEDIKIQFSRSGSGSGGFLEGLFGCLRPVWNIIGKAYSTDYKLQQQDTWEVPFEEISELQWLGSGAQGAVFLGKFRAEEVAIKKVREQNETDIKHLRKLKHPNIIAFKGVCTQAPCYCIIMEYCAHGQLYEVLRAGRKITPRLLVDWSTGIASGMNYLHLHKIIHRDLKSPNVLVTHTDAVKISDFGTSKELSDKSTKMSFAGTVAWMAPEVIRNEPVSEKVDIWSFGVVLWELLTGEIPYKDVDSSAIIWGVGSNSLHLPVPSTCPDGFKILMKQTWQSKPRNRPSFRQTLMHLDIASADVLATPQETYFKSQAEWREEVKKHFEKIKSEGTCIHRLDEELIRRRREELRHALDIREHYERKLERANNLYMELSAIMLQLEMREKELIKREQAVEKKYPGTYKRHPVRPIIHPNAMEKLMKRKGVPHKSGMQTKRPDLLRSEGIPTTEVAPTASPLSGSPKMSTSSSKSRYRSKPRHRRGNSRGSHSDFAAILKNQPAQENSPHPTYLHQAQSQYPSLHHHNSLQQQYQQPPPAMSQSHHPRLNMHGQDIATCANNLRYFGPAAALRSPLSNHAQRQLPGSSPDLISTAMAADCWRSSEPDKGQAGPWGCCQADAYDPCLQCRPEQYGSLDIPSAEPVGRSPDLSKSPAHNPLLENAQSSEKTEENEFSGCRSESSLGTSHLGTPPALPRKTRPLQKSGDDSSEEEEGEVDSEVEFPRRQRPHRCISSCQSYSTFSSENFSVSDGEEGNTSDHSNSPDELADKLEDRLAEKLDDLLSQTPEIPIDISSHSDGLSDKECAVRRVKTQMSLGKLCVEERGYENPMQFEESDCDSSDGECSDATVRTNKHYSSATW",
# DDR1_HUMAN
"MGPEALSSLLLLLLVASGDADMKGHFDPAKCRYALGMQDRTIPDSDISASSSWSDSTAARHSRLESSDGDGAWCPAGSVFPKEEEYLQVDLQRLHLVALVGTQGRHAGGLGKEFSRSYRLRYSRDGRRWMGWKDRWGQEVISGNEDPEGVVLKDLGPPMVARLVRFYPRADRVMSVCLRVELYGCLWRDGLLSYTAPVGQTMYLSEAVYLNDSTYDGHTVGGLQYGGLGQLADGVVGLDDFRKSQELRVWPGYDYVGWSNHSFSSGYVEMEFEFDRLRAFQAMQVHCNNMHTLGARLPGGVECRFRRGPAMAWEGEPMRHNLGGNLGDPRARAVSVPLGGRVARFLQCRFLFAGPWLLFSEISFISDVVNNSSPALGGTFPPAPWWPPGPPPTNFSSLELEPRGQQPVAKAEGSPTAILIGCLVAIILLLLLIIALMLWRLHWRRLLSKAERRVLEEELTVHLSVPGDTILINNRPGPREPPPYQEPRPRGNPPHSAPCVPNGSALLLSNPAYRLLLATYARPPRGPGPPTPAWAKPTNTQAYSGDYMEPEKPGAPLLPPPPQNSVPHYAEADIVTLQGVTGGNTYAVPALPPGAVGDGPPRVDFPRSRLRFKEKLGEGQFGEVHLCEVDSPQDLVSLDFPLNVRKGHPLLVAVKILRPDATKNARNDFLKEVKIMSRLKDPNIIRLLGVCVQDDPLCMITDYMENGDLNQFLSAHQLEDKAAEGAPGDGQAAQGPTISYPMLLHVAAQIASGMRYLATLNFVHRDLATRNCLVGENFTIKIADFGMSRNLYAGDYYRVQGRAVLPIRWMAWECILMGKFTTASDVWAFGVTLWEVLMLCRAQPFGQLTDEQVIENAGEFFRDQGRQVYLSRPPACPQGLYELMLRCWSRESEQRPPFSQLHRFLAEDALNTV",
# ADCK1_HUMAN
"MARKALKLASWTSMALAASGIYFYSNKYLDPNDFGAVRVGRAVATTAVISYDYLTSLKSVPYGSEEYLQLRSKSWPVFLQVHLRSARRLCELCCANRGTFIKVGQHLGALDYLLPEEYTSTLKVLHSQAPQSSMQEIRQVIREDLGKEIHDLFQSFDDTPLGTASLAQVHKAVLHDGRTVAVKVQHPKVRAQSSKDILLMEVLVLAVKQLFPEFEFMWLVDEAKKNLPLELDFLNEGRNAEKVSQMLRHFDFLKVPRIHWDLSTERVLLMEFVDGGQVNDRDYMERNKIDVNEISRHLGKMYSEMIFVNGFVHCDPHPGNVLVRKHPGTGKAEIVLLDHGLYQMLTEEFRLNYCHLWQSLIWTDMKRVKEYSQRLGAGDLYPLFACMLTARSWDSVNRGISQAPVTATEDLEIRNNAANYLPQISHLLNHVPRQMLLILKTNDLLRGIEAALGTRASASSFLNMSRCCIRALAEHKKKNTCSFFRRTQISFSEAFNLWQINLHELILRVKGLKLADRVLALICWLFPAPL",
# VGFR2_HUMAN
"MQSKVLLAVALWLCVETRAASVGLPSVSLDLPRLSIQKDILTIKANTTLQITCRGQRDLDWLWPNNQSGSEQRVEVTECSDGLFCKTLTIPKVIGNDTGAYKCFYRETDLASVIYVYVQDYRSPFIASVSDQHGVVYITENKNKTVVIPCLGSISNLNVSLCARYPEKRFVPDGNRISWDSKKGFTIPSYMISYAGMVFCEAKINDESYQSIMYIVVVVGYRIYDVVLSPSHGIELSVGEKLVLNCTARTELNVGIDFNWEYPSSKHQHKKLVNRDLKTQSGSEMKKFLSTLTIDGVTRSDQGLYTCAASSGLMTKKNSTFVRVHEKPFVAFGSGMESLVEATVGERVRIPAKYLGYPPPEIKWYKNGIPLESNHTIKAGHVLTIMEVSERDTGNYTVILTNPISKEKQSHVVSLVVYVPPQIGEKSLISPVDSYQYGTTQTLTCTVYAIPPPHHIHWYWQLEEECANEPSQAVSVTNPYPCEEWRSVEDFQGGNKIEVNKNQFALIEGKNKTVSTLVIQAANVSALYKCEAVNKVGRGERVISFHVTRGPEITLQPDMQPTEQESVSLWCTADRSTFENLTWYKLGPQPLPIHVGELPTPVCKNLDTLWKLNATMFSNSTNDILIMELKNASLQDQGDYVCLAQDRKTKKRHCVVRQLTVLERVAPTITGNLENQTTSIGESIEVSCTASGNPPPQIMWFKDNETLVEDSGIVLKDGNRNLTIRRVRKEDEGLYTCQACSVLGCAKVEAFFIIEGAQEKTNLEIIILVGTAVIAMFFWLLLVIILRTVKRANGGELKTGYLSIVMDPDELPLDEHCERLPYDASKWEFPRDRLKLGKPLGRGAFGQVIEADAFGIDKTATCRTVAVKMLKEGATHSEHRALMSELKILIHIGHHLNVVNLLGACTKPGGPLMVIVEFCKFGNLSTYLRSKRNEFVPYKTKGARFRQGKDYVGAIPVDLKRRLDSITSSQSSASSGFVEEKSLSDVEEEEAPEDLYKDFLTLEHLICYSFQVAKGMEFLASRKCIHRDLAARNILLSEKNVVKICDFGLARDIYKDPDYVRKGDARLPLKWMAPETIFDRVYTIQSDVWSFGVLLWEIFSLGASPYPGVKIDEEFCRRLKEGTRMRAPDYTTPEMYQTMLDCWHGEPSQRPTFSELVEHLGNLLQANAQQDGKDYIVLPISETLSMEEDSGLSLPTSPVSCMEEEEVCDPKFHYDNTAGISQYLQNSKRKSRPVSVKTFEDIPLEEPEVKVIPDDNQTDSGMVLASEELKTLEDRTKLSPSFGGMVPSKSRESVASEGSNQTSGYQSGYHSDDTDTTVYSSEEAELLKLIEIGVQTGSTAQILQPDSGTTLSSPPV",
# ACV1C_HUMAN
"MTRALCSALRQALLLLAAAAELSPGLKCVCLLCDSSNFTCQTEGACWASVMLTNGKEQVIKSCVSLPELNAQVFCHSSNNVTKTECCFTDFCNNITLHLPTASPNAPKLGPMELAIIITVPVCLLSIAAMLTVWACQGRQCSYRKKKRPNVEEPLSECNLVNAGKTLKDLIYDVTASGSGSGLPLLVQRTIARTIVLQEIVGKGRFGEVWHGRWCGEDVAVKIFSSRDERSWFREAEIYQTVMLRHENILGFIAADNKDNGTWTQLWLVSEYHEQGSLYDYLNRNIVTVAGMIKLALSIASGLAHLHMEIVGTQGKPAIAHRDIKSKNILVKKCETCAIADLGLAVKHDSILNTIDIPQNPKVGTKRYMAPEMLDDTMNVNIFESFKRADIYSVGLVYWEIARRCSVGGIVEEYQLPYYDMVPSDPSIEEMRKVVCDQKFRPSIPNQWQSCEALRVMGRIMRECWYANGAARLTALRIKKTISQLCVKEDCKA",
# AURKB_HUMAN
"MAQKENSYPWPYGRQTAPSGLSTLPQRVLRKEPVTPSALVLMSRSNVQPTAAPGQKVMENSSGTPDILTRHFTIDDFEIGRPLGKGKFGNVYLAREKKSHFIVALKVLFKSQIEKEGVEHQLRREIEIQAHLHHPNILRLYNYFYDRRRIYLILEYAPRGELYKELQKSCTFDEQRTATIMEELADALMYCHGKKVIHRDIKPENLLLGLKGELKIADFGWSVHAPSLRRKTMCGTLDYLPPEMIEGRMHNEKVDLWCIGVLCYELLVGNPPFESASHNETYRRIVKVDLKFPASVPMGAQDLISKLLRHNPSERLPLAQVSAHPWVRANSRRVLPPSALQSVA",
# AURKA_HUMAN
"MDRSKENCISGPVKATAPVGGPKRVLVTQQFPCQNPLPVNSGQAQRVLCPSNSSQRIPLQAQKLVSSHKPVQNQKQKQLQATSVPHPVSRPLNNTQKSKQPLPSAPENNPEEELASKQKNEESKKRQWALEDFEIGRPLGKGKFGNVYLAREKQSKFILALKVLFKAQLEKAGVEHQLRREVEIQSHLRHPNILRLYGYFHDATRVYLILEYAPLGTVYRELQKLSKFDEQRTATYITELANALSYCHSKRVIHRDIKPENLLLGSAGELKIADFGWSVHAPSSRRTTLCGTLDYLPPEMIEGRMHDEKVDLWSLGVLCYEFLVGKPPFEANTYQETYKRISRVEFTFPDFVTEGARDLISRLLKHNPSQRPMLREVLEHPWITANSSKPSNCQNKESASKQS",
# MK07_HUMAN
"MAEPLKEEDGEDGSAEPPGPVKAEPAHTAASVAAKNLALLKARSFDVTFDVGDEYEIIETIGNGAYGVVSSARRRLTGQQVAIKKIPNAFDVVTNAKRTLRELKILKHFKHDNIIAIKDILRPTVPYGEFKSVYVVLDLMESDLHQIIHSSQPLTLEHVRYFLYQLLRGLKYMHSAQVIHRDLKPSNLLVNENCELKIGDFGMARGLCTSPAEHQYFMTEYVATRWYRAPELMLSLHEYTQAIDLWSVGCIFGEMLARRQLFPGKNYVHQLQLIMMVLGTPSPAVIQAVGAERVRAYIQSLPPRQPVPWETVYPGADRQALSLLGRMLRFEPSARISAAAALRHPFLAKYHDPDDEPDCAPPFDFAFDREALTRERIKEAIVAEIEDFHARREGIRQQIRFQPSLQPVASEPGCPDVEMPSPWAPSGDCAMESPPPAPPPCPGPAPDTIDLTLQPPPPVSEPAPPKKDGAISDNTKAALKAALLKSLRSRLRDGPSAPLEAPEPRKPVTAQERQREREEKRRRRQERAKEREKRRQERERKERGAGASGGPSTDPLAGLVLSDNDRSLLERWTRMARPAAPALTSVPAPAPAPTPTPTPVQPTSPPPGPVAQPTGPQPQSAGSTSGPVPQPACPPPGPAPHPTGPPGPIPVPAPPQIATSTSLLAAQSLVPPPGLPGSSTPGVLPYFPPGLPPPDAGGAPQSSMSESPDVNLVTQQLSKSQVEDPLPPVFSGTPKGSGAGYGVGFDLEEFLNQSFDMGVADGPQDGQADSASLSASLLADWLEGHGMNPADIESLQREIQMDSPMLLADLPDLQDP",
# BUB1_HUMAN
"MDTPENVLQMLEAHMQSYKGNDPLGEWERYIQWVEENFPENKEYLITLLEHLMKEFLDKKKYHNDPRFISYCLKFAEYNSDLHQFFEFLYNHGIGTLSSPLYIAWAGHLEAQGELQHASAVLQRGIQNQAEPREFLQQQYRLFQTRLTETHLPAQARTSEPLHNVQVLNQMITSKSNPGNNMACISKNQGSELSGVISSACDKESNMERRVITISKSEYSVHSSLASKVDVEQVVMYCKEKLIRGESEFSFEELRAQKYNQRRKHEQWVNEDRHYMKRKEANAFEEQLLKQKMDELHKKLHQVVETSHEDLPASQERSEVNPARMGPSVGSQQELRAPCLPVTYQQTPVNMEKNPREAPPVVPPLANAISAALVSPATSQSIAPPVPLKAQTVTDSMFAVASKDAGCVNKSTHEFKPQSGAEIKEGCETHKVANTSSFHTTPNTSLGMVQATPSKVQPSPTVHTKEALGFIMNMFQAPTLPDISDDKDEWQSLDQNEDAFEAQFQKNVRSSGAWGVNKIISSLSSAFHVFEDGNKENYGLPQPKNKPTGARTFGERSVSRLPSKPKEEVPHAEEFLDDSTVWGIRCNKTLAPSPKSPGDFTSAAQLASTPFHKLPVESVHILEDKENVVAKQCTQATLDSCEENMVVPSRDGKFSPIQEKSPKQALSSHMYSASLLRLSQPAAGGVLTCEAELGVEACRLTDTDAAIAEDPPDAIAGLQAEWMQMSSLGTVDAPNFIVGNPWDDKLIFKLLSGLSKPVSSYPNTFEWQCKLPAIKPKTEFQLGSKLVYVHHLLGEGAFAQVYEATQGDLNDAKNKQKFVLKVQKPANPWEFYIGTQLMERLKPSMQHMFMKFYSAHLFQNGSVLVGELYSYGTLLNAINLYKNTPEKVMPQGLVISFAMRMLYMIEQVHDCEIIHGDIKPDNFILGNGFLEQDDEDDLSAGLALIDLGQSIDMKLFPKGTIFTAKCETSGFQCVEMLSNKPWNYQIDYFGVAATVYCMLFGTYMKVKNEGGECKPEGLFRRLPHLDMWNEFFHVMLNIPDCHHLPSLDLLRQKLKKVFQQHYTNKIRALRNRLIVLLLECKRSRK",
# DDR2_HUMAN
"MILIPRMLLVLFLLLPILSSAKAQVNPAICRYPLGMSGGQIPDEDITASSQWSESTAAKYGRLDSEEGDGAWCPEIPVEPDDLKEFLQIDLHTLHFITLVGTQGRHAGGHGIEFAPMYKINYSRDGTRWISWRNRHGKQVLDGNSNPYDIFLKDLEPPIVARFVRFIPVTDHSMNVCMRVELYGCVWLDGLVSYNAPAGQQFVLPGGSIIYLNDSVYDGAVGYSMTEGLGQLTDGVSGLDDFTQTHEYHVWPGYDYVGWRNESATNGYIEIMFEFDRIRNFTTMKVHCNNMFAKGVKIFKEVQCYFRSEASEWEPNAISFPLVLDDVNPSARFVTVPLHHRMASAIKCQYHFADTWMMFSEITFQSDAAMYNNSEALPTSPMAPTTYDPMLKVDDSNTRILIGCLVAIIFILLAIIVIILWRQFWQKMLEKASRRMLDDEMTVSLSLPSDSSMFNNNRSSSPSEQGSNSTYDRIFPLRPDYQEPSRLIRKLPEFAPGEEESGCSGVVKPVQPSGPEGVPHYAEADIVNLQGVTGGNTYSVPAVTMDLLSGKDVAVEEFPRKLLTFKEKLGEGQFGEVHLCEVEGMEKFKDKDFALDVSANQPVLVAVKMLRADANKNARNDFLKEIKIMSRLKDPNIIHLLAVCITDDPLCMITEYMENGDLNQFLSRHEPPNSSSSDVRTVSYTNLKFMATQIASGMKYLSSLNFVHRDLATRNCLVGKNYTIKIADFGMSRNLYSGDYYRIQGRAVLPIRWMSWESILLGKFTTASDVWAFGVTLWETFTFCQEQPYSQLSDEQVIENTGEFFRDQGRQTYLPQPAICPDSVYKLMLSCWRRDTKNRPSFQEIHLLLLQQGDE",
# PTK7_HUMAN
"MGAARGSPARPRRLPLLSVLLLPLLGGTQTAIVFIKQPSSQDALQGRRALLRCEVEAPGPVHVYWLLDGAPVQDTERRFAQGSSLSFAAVDRLQDSGTFQCVARDDVTGEEARSANASFNIKWIEAGPVVLKHPASEAEIQPQTQVTLRCHIDGHPRPTYQWFRDGTPLSDGQSNHTVSSKERNLTLRPAGPEHSGLYSCCAHSAFGQACSSQNFTLSIADESFARVVLAPQDVVVARYEEAMFHCQFSAQPPPSLQWLFEDETPITNRSRPPHLRRATVFANGSLLLTQVRPRNAGIYRCIGQGQRGPPIILEATLHLAEIEDMPLFEPRVFTAGSEERVTCLPPKGLPEPSVWWEHAGVRLPTHGRVYQKGHELVLANIAESDAGVYTCHAANLAGQRRQDVNITVATVPSWLKKPQDSQLEEGKPGYLDCLTQATPKPTVVWYRNQMLISEDSRFEVFKNGTLRINSVEVYDGTWYRCMSSTPAGSIEAQARVQVLEKLKFTPPPQPQQCMEFDKEATVPCSATGREKPTIKWERADGSSLPEWVTDNAGTLHFARVTRDDAGNYTCIASNGPQGQIRAHVQLTVAVFITFKVEPERTTVYQGHTALLQCEAQGDPKPLIQWKGKDRILDPTKLGPRMHIFQNGSLVIHDVAPEDSGRYTCIAGNSCNIKHTEAPLYVVDKPVPEESEGPGSPPPYKMIQTIGLSVGAAVAYIIAVLGLMFYCKKRCKAKRLQKQPEGEEPEMECLNGGPLQNGQPSAEIQEEVALTSLGSGPAATNKRHSTSDKMHFPRSSLQPITTLGKSEFGEVFLAKAQGLEEGVAETLVLVKSLQSKDEQQQLDFRRELEMFGKLNHANVVRLLGLCREAEPHYMVLEYVDLGDLKQFLRISKSKDEKLKSQPLSTKQKVALCTQVALGMEHLSNNRFVHKDLAARNCLVSAQRQVKVSALGLSKDVYNSEYYHFRQAWVPLRWMSPEAILEGDFSTKSDVWAFGVLMWEVFTHGEMPHGGQADDEVLADLQAGKARLPQPEGCPSKLYRLMQRCWALSPKDRPSFSEIASALGDSTVDSKP",
# LIMK1_HUMAN
"MRLTLLCCTWREERMGEEGSELPVCASCGQRIYDGQYLQALNADWHADCFRCCDCSASLSHQYYEKDGQLFCKKDYWARYGESCHGCSEQITKGLVMVAGELKYHPECFICLTCGTFIGDGDTYTLVEHSKLYCGHCYYQTVVTPVIEQILPDSPGSHLPHTVTLVSIPASSHGKRGLSVSIDPPHGPPGCGTEHSHTVRVQGVDPGCMSPDVKNSIHVGDRILEINGTPIRNVPLDEIDLLIQETSRLLQLTLEHDPHDTLGHGLGPETSPLSSPAYTPSGEAGSSARQKPVLRSCSIDRSPGAGSLGSPASQRKDLGRSESLRVVCRPHRIFRPSDLIHGEVLGKGCFGQAIKVTHRETGEVMVMKELIRFDEETQRTFLKEVKVMRCLEHPNVLKFIGVLYKDKRLNFITEYIKGGTLRGIIKSMDSQYPWSQRVSFAKDIASGMAYLHSMNIIHRDLNSHNCLVRENKNVVVADFGLARLMVDEKTQPEGLRSLKKPDRKKRYTVVGNPYWMAPEMINGRSYDEKVDVFSFGIVLCEIIGRVNADPDYLPRTMDFGLNVRGFLDRYCPPNCPPSFFPITVRCCDLDPEKRPSFVKLEHWLETLRMHLAGHLPLGPQLEQLDRGFWETYRRGESGLPAHPEVPD",
# LMTK1_HUMAN
"MSSSFFNPSFAFSSHFDPDGAPLSELSWPSSLAVVAVSFSGLFAVIVLMLACLCCKKGGIGFKEFENAEGDEYAADLAQGSPATAAQNGPDVYVLPLTEVSLPMAKQPGRSVQLLKSTDVGRHSLLYLKEIGRGWFGKVFLGEVNSGISSAQVVVKELQASASVQEQMQFLEEVQPYRALKHSNLLQCLAQCAEVTPYLLVMEFCPLGDLKGYLRSCRVAESMAPDPRTLQRMACEVACGVLHLHRNNFVHSDLALRNCLLTADLTVKIGDYGLAHCKYREDYFVTADQLWVPLRWIAPELVDEVHSNLLVVDQTKSGNVWSLGVTIWELFELGTQPYPQHSDQQVLAYTVREQQLKLPKPQLQLTLSDRWYEVMQFCWLQPEQRPTAEEVHLLLSYLCAKGATEAEEEFERRWRSLRPGGGGVGPGPGAAGPMLGGVVELAAASSFPLLEQFAGDGFHADGDDVLTVTETSRGLNFEYKWEAGRGAEAFPATLSPGRTARLQELCAPDGAPPGVVPVLSAHSPSLGSEYFIRLEEAAPAAGHDPDCAGCAPSPPATADQDDDSDGSTAASLAMEPLLGHGPPVDVPWGRGDHYPRRSLARDPLCPSRSPSPSAGPLSLAEGGAEDADWGVAAFCPAFFEDPLGTSPLGSSGAPPLPLTGEDELEEVGARRAAQRGHWRSNVSANNNSGSRCPESWDPVSAGGHAEGCPSPKQTPRASPEPGYPGEPLLGLQAASAQEPGCCPGLPHLCSAQGLAPAPCLVTPSWTETASSGGDHPQAEPKLATEAEGTTGPRLPLPSVPSPSQEGAPLPSEEASAPDAPDALPDSPTPATGGEVSAIKLASALNGSSSSPEVEAPSSEDEDTAEATSGIFTDTSSDGLQARRPDVVPAFRSLQKQVGTPDSLDSLDIPSSASDGGYEVFSPSATGPSGGQPRALDSGYDTENYESPEFVLKEAQEGCEPQAFAELASEGEGPGPETRLSTSLSGLNEKNPYRDSAYFSDLEAEAEATSGPEKKCGGDRAPGPELGLPSTGQPSEQVCLRPGVSGEAQGSGPGEVLPPLLQLEGSSPEPSTCPSGLVPEPPEPQGPAKVRPGPSPSCSQFFLLTPVPLRSEGNSSEFQGPPGLLSGPAPQKRMGGPGTPRAPLRLALPGLPAALEGRPEEEEEDSEDSDESDEELRCYSVQEPSEDSEEEAPAVPVVVAESQSARNLRSLLKMPSLLSETFCEDLERKKKAVSFFDDVTVYLFDQESPTRELGEPFPGAKESPPTFLRGSPGSPSAPNRPQQADGSPNGSTAEEGGGFAWDDDFPLMTAKAAFAMALDPAAPAPAAPTPTPAPFSRFTVSPAPTSRFSITHVSDSDAESKRGPEAGAGGESKEA",
# LMTK2_HUMAN
"MPGPPALRRRLLLLLLVLLIAGSAGAAPLPQTGAGEAPPAAEVSSSFVILCVCSLIILIVLIANCVSCCKDPEIDFKEFEDNFDDEIDFTPPAEDTPSVQSPAEVFTLSVPNISLPAPSQFQPSVEGLKSQVARHSLNYIQEIGNGWFGKVLLGEIYTGTSVARVIVKELKASANPKEQDTFLKNGEPYYILQHPNILQCVGQCVEAIPYLLVFEFCDLGDLKAYLRSEQEHMRGDSQTMLLQRMACEVAAGLAAMHKLHFLHSDLALRNCFLTSDLNVKVGDYGIGFSRYKEDYIETDDKKVFPLRWTAPELVTSFQDRLLTADQTKYSNIWSLGVTLWELFDNAAQPYSNLSNLDVLNQVIRERDTKLPKPQLEQPYSDRWYEVLQFCWLSPEKRPAAEDVHRLLTYLRLQSQRDSEVDFEQQWNALKPNTNSRDSSNNAAFPILDHFARDRLGREMEEVLTVTETSQGLSFEYVWEAAKHDHFDERSRGHLDEGLSYTSIFYPVEVFESSLSDPGPGKQDDSGQDVPLRVPGVVPVFDAHNLSVGSDYYIQLEEKSGSNLELDYPPALLTTDMDNPERTGPELSQLTALRSVELEESSTDEDFFQSSTDPKDSSLPGDLHVTSGPESPFNNIFNDVDKSEDLPSHQKIFDLMELNGVQADFKPATLSSSLDNPKESVITGHFEKEKPRKIFDSEPLCLSDNLMHQDNFDPLNVQELSENFLFLQEKNLLKGSLSSKEHINDLQTELKNAGFTEAMLETSCRNSLDTELQFAENKPGLSLLQENVSTKGDDTDVMLTGDTLSTSLQSSPEVQVPPTSFETEETPRRVPPDSLPTQGETQPTCLDVIVPEDCLHQDISPDAVTVPVEILSTDARTHSLDNRSQDSPGESEETLRLTESDSVLADDILASRVSVGSSLPELGQELHNKPFSEDHHSHRRLEKNLEAVETLNQLNSKDAAKEAGLVSALSSDSTSQDSLLEDSLSAPFPASEPSLETPDSLESVDVHEALLDSLGSHTPQKLVPPDKPADSGYETENLESPEWTLHPAPEGTADSEPATTGDGGHSGLPPNPVIVISDAGDGHRGTEVTPETFTAGSQGSYRDSAYFSDNDSEPEKRSEEVPGTSPSALVLVQEQPLPEPVLPEQSPAAQDSCLEARKSQPDESCLSALHNSSDLELRATPEPAQTGVPQQVHPTEDEASSPWSVLNAELSSGDDFETQDDRPCTLASTGTNTNELLAYTNSALDKSLSSHSEGPKLKEPDIEGKYLGKLGVSGMLDLSEDGMDADEEDENSDDSDEDLRAFNLHSLSSESEDETEHPVPIILSNEDGRHLRSLLKPTAANAPDPLPEDWKKEKKAVTFFDDVTVYLFDQETPTKELGPCGGEACGPDLSGPAPASGSPYLSRCINSESSTDEEGGGFEWDDDFSPDPFMSKTTSNLLSSKPSLQTSKYFSPPPPARSTEQSWPHSAPYSRFSISPANIASFSLTHLTDSDIEQGGSSEDGEKD",
# LMTK3_HUMAN
"MPAPGALILLAAVSASGCLASPAHPDGFALGRAPLAPPYAVVLISCSGLLAFIFLLLTCLCCKRGDVGFKEFENPEGEDCSGEYTPPAEETSSSQSLPDVYILPLAEVSLPMPAPQPSHSDMTTPLGLSRQHLSYLQEIGSGWFGKVILGEIFSDYTPAQVVVKELRASAGPLEQRKFISEAQPYRSLQHPNVLQCLGLCVETLPFLLIMEFCQLGDLKRYLRAQRPPEGLSPELPPRDLRTLQRMGLEIARGLAHLHSHNYVHSDLALRNCLLTSDLTVRIGDYGLAHSNYKEDYYLTPERLWIPLRWAAPELLGELHGTFMVVDQSRESNIWSLGVTLWELFEFGAQPYRHLSDEEVLAFVVRQQHVKLARPRLKLPYADYWYDILQSCWRPPAQRPSASDLQLQLTYLLSERPPRPPPPPPPPRDGPFPWPWPPAHSAPRPGTLSSPFPLLDGFPGADPDDVLTVTESSRGLNLECLWEKARRGAGRGGGAPAWQPASAPPAPHANPSNPFYEALSTPSVLPVISARSPSVSSEYYIRLEEHGSPPEPLFPNDWDPLDPGVPAPQAPQAPSEVPQLVSETWASPLFPAPRPFPAQSSASGSFLLSGWDPEGRGAGETLAGDPAEVLGERGTAPWVEEEEEEEEGSSPGEDSSSLGGGPSRRGPLPCPLCSREGACSCLPLERGDAVAGWGGHPALGCPHPPEDDSSLRAERGSLADLPMAPPASAPPEFLDPLMGAAAPQYPGRGPPPAPPPPPPPPRAPADPAASPDPPSAVASPGSGLSSPGPKPGDSGYETETPFSPEGAFPGGGAAEEEGVPRPRAPPEPPDPGAPRPPPDPGPLPLPGPREKPTFVVQVSTEQLLMSLREDVTRNLLGEKGATARETGPRKAGRGPGNREKVPGLNRDPTVLGNGKQAPSLSLPVNGVTVLENGDQRAPGIEEKAAENGALGSPEREEKVLENGELTPPRREEKALENGELRSPEAGEKVLVNGGLTPPKSEDKVSENGGLRFPRNTERPPETGPWRAPGPWEKTPESWGPAPTIGEPAPETSLERAPAPSAVVSSRNGGETAPGPLGPAPKNGTLEPGTERRAPETGGAPRAPGAGRLDLGSGGRAPVGTGTAPGGGPGSGVDAKAGWVDNTRPQPPPPPLPPPPEAQPRRLEPAPPRARPEVAPEGEPGAPDSRAGGDTALSGDGDPPKPERKGPEMPRLFLDLGPPQGNSEQIKARLSRLSLALPPLTLTPFPGPGPRRPPWEGADAGAAGGEAGGAGAPGPAEEDGEDEDEDEEEDEEAAAPGAAAGPRGPGRARAAPVPVVVSSADADAARPLRGLLKSPRGADEPEDSELERKRKMVSFHGDVTVYLFDQETPTNELSVQAPPEGDTDPSTPPAPPTPPHPATPGDGFPSNDSGFGGSFEWAEDFPLLPPPGPPLCFSRFSVSPALETPGPPARAPDARPAGPVEN",
# EPHA7_HUMAN
"MVFQTRYPSWIILCYIWLLRFAHTGEAQAAKEVLLLDSKAQQTELEWISSPPNGWEEISGLDENYTPIRTYQVCQVMEPNQNNWLRTNWISKGNAQRIFVELKFTLRDCNSLPGVLGTCKETFNLYYYETDYDTGRNIRENLYVKIDTIAADESFTQGDLGERKMKLNTEVREIGPLSKKGFYLAFQDVGACIALVSVKVYYKKCWSIIENLAIFPDTVTGSEFSSLVEVRGTCVSSAEEEAENAPRMHCSAEGEWLVPIGKCICKAGYQQKGDTCEPCGRGFYKSSSQDLQCSRCPTHSFSDKEGSSRCECEDGYYRAPSDPPYVACTRPPSAPQNLIFNINQTTVSLEWSPPADNGGRNDVTYRILCKRCSWEQGECVPCGSNIGYMPQQTGLEDNYVTVMDLLAHANYTFEVEAVNGVSDLSRSQRLFAAVSITTGQAAPSQVSGVMKERVLQRSVELSWQEPEHPNGVITEYEIKYYEKDQRERTYSTVKTKSTSASINNLKPGTVYVFQIRAFTAAGYGNYSPRLDVATLEEATGKMFEATAVSSEQNPVIIIAVVAVAGTIILVFMVFGFIIGRRHCGYSKADQEGDEELYFHFKFPGTKTYIDPETYEDPNRAVHQFAKELDASCIKIERVIGAGEFGEVCSGRLKLPGKRDVAVAIKTLKVGYTEKQRRDFLCEASIMGQFDHPNVVHLEGVVTRGKPVMIVIEFMENGALDAFLRKHDGQFTVIQLVGMLRGIAAGMRYLADMGYVHRDLAARNILVNSNLVCKVSDFGLSRVIEDDPEAVYTTTGGKIPVRWTAPEAIQYRKFTSASDVWSYGIVMWEVMSYGERPYWDMSNQDVIKAIEEGYRLPAPMDCPAGLHQLMLDCWQKERAERPKFEQIVGILDKMIRNPNSLKTPLGTCSRPISPLLDQNTPDFTTFCSVGEWLQAIKMERYKDNFTAAGYNSLESVARMTIEDVMSLGITLVGHQKKIMSSIQTMRAQMLHLHGTGIQV",
# BMX_HUMAN
"MDTKSILEELLLKRSQQKKKMSPNNYKERLFVLTKTNLSYYEYDKMKRGSRKGSIEIKKIRCVEKVNLEEQTPVERQYPFQIVYKDGLLYVYASNEESRSQWLKALQKEIRGNPHLLVKYHSGFFVDGKFLCCQQSCKAAPGCTLWEAYANLHTAVNEEKHRVPTFPDRVLKIPRAVPVLKMDAPSSSTTLAQYDNESKKNYGSQPPSSSTSLAQYDSNSKKIYGSQPNFNMQYIPREDFPDWWQVRKLKSSSSSEDVASSNQKERNVNHTTSKISWEFPESSSSEEEENLDDYDWFAGNISRSQSEQLLRQKGKEGAFMVRNSSQVGMYTVSLFSKAVNDKKGTVKHYHVHTNAENKLYLAENYCFDSIPKLIHYHQHNSAGMITRLRHPVSTKANKVPDSVSLGNGIWELKREEITLLKELGSGQFGVVQLGKWKGQYDVAVKMIKEGSMSEDEFFQEAQTMMKLSHPKLVKFYGVCSKEYPIYIVTEYISNGCLLNYLRSHGKGLEPSQLLEMCYDVCEGMAFLESHQFIHRDLAARNCLVDRDLCVKVSDFGMTRYVLDDQYVSSVGTKFPVKWSAPEVFHYFKYSSKSDVWAFGILMWEVFSLGKQPYDLYDNSQVVLKVSQGHRLYRPHLASDTIYQIMYSCWHELPEKRPTFQQLLSSIEPLREKDKH",
# MATK_HUMAN
"MAGRGSLVSWRAFHGCDSAEELPRVSPRFLRAWHPPPVSARMPTRRWAPGTQCITKCEHTRPKPGELAFRKGDVVTILEACENKSWYRVKHHTSGQEGLLAAGALREREALSADPKLSLMPWFHGKISGQEAVQQLQPPEDGLFLVRESARHPGDYVLCVSFGRDVIHYRVLHRDGHLTIDEAVFFCNLMDMVEHYSKDKGAICTKLVRPKRKHGTKSAEEELARAGWLLNLQHLTLGAQIGEGEFGAVLQGEYLGQKVAVKNIKCDVTAQAFLDETAVMTKMQHENLVRLLGVILHQGLYIVMEHVSKGNLVNFLRTRGRALVNTAQLLQFSLHVAEGMEYLESKKLVHRDLAARNILVSEDLVAKVSDFGLAKAERKGLDSSRLPVKWTAPEALKHGKFTSKSDVWSFGVLLWEVFSYGRAPYPKMSLKEVSEAVEKGYRMEPPEGCPGPVHVLMSSCWEAEPARRPPFRKLAEKLARELRSAGAPASVSGQDADGSTSPRSQEP",
# FRK_HUMAN
"MSNICQRLWEYLEPYLPCLSTEADKSTVIENPGALCSPQSQRHGHYFVALFDYQARTAEDLSFRAGDKLQVLDTLHEGWWFARHLEKRRDGSSQQLQGYIPSNYVAEDRSLQAEPWFFGAIGRSDAEKQLLYSENKTGSFLIRESESQKGEFSLSVLDGAVVKHYRIKRLDEGGFFLTRRRIFSTLNEFVSHYTKTSDGLCVKLGKPCLKIQVPAPFDLSYKTVDQWEIDRNSIQLLKRLGSGQFGEVWEGLWNNTTPVAVKTLKPGSMDPNDFLREAQIMKNLRHPKLIQLYAVCTLEDPIYIITELMRHGSLQEYLQNDTGSKIHLTQQVDMAAQVASGMAYLESRNYIHRDLAARNVLVGEHNIYKVADFGLARVFKVDNEDIYESRHEIKLPVKWTAPEAIRSNKFSIKSDVWSFGILLYEIITYGKMPYSGMTGAQVIQMLAQNYRLPQPSNCPQQFYNIMLECWNAEPKERPTFETLRWKLEDYFETDSSYSDANNFIR",
# NEK6_HUMAN
"MAGQPGHMPHGGSSNNLCHTLGPVHPPDPQRHPNTLSFRCSLADFQIEKKIGRGQFSEVYKATCLLDRKTVALKKVQIFEMMDAKARQDCVKEIGLLKQLNHPNIIKYLDSFIEDNELNIVLELADAGDLSQMIKYFKKQKRLIPERTVWKYFVQLCSAVEHMHSRRVMHRDIKPANVFITATGVVKLGDLGLGRFFSSETTAAHSLVGTPYYMSPERIHENGYNFKSDIWSLGCLLYEMAALQSPFYGDKMNLFSLCQKIEQCDYPPLPGEHYSEKLRELVSMCICPDPHQRPDIGYVHQVAKQMHIWMSST",
# NEK7_HUMAN
"MDEQSQGMQGPPVPQFQPQKALRPDMGYNTLANFRIEKKIGRGQFSEVYRAACLLDGVPVALKKVQIFDLMDAKARADCIKEIDLLKQLNHPNVIKYYASFIEDNELNIVLELADAGDLSRMIKHFKKQKRLIPERTVWKYFVQLCSALEHMHSRRVMHRDIKPANVFITATGVVKLGDLGLGRFFSSKTTAAHSLVGTPYYMSPERIHENGYNFKSDIWSLGCLLYEMAALQSPFYGDKMNLYSLCKKIEQCDYPPLPSDHYSEELRQLVNMCINPDPEKRPDVTYVYDVAKRMHACTASS",
# AAK1_HUMAN
"MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGEDSLLDCSLLSNPTTDLLEEFAPTAISAPVHKAAEDSNLISGFDVPEGSDKVAEDEFDPIPVLITKNPQGGHSRNSSGSSESSLPNLARSLLLVDQLIDL",
# TRPM7_HUMAN
"MSQKSWIESTLTKRECVYIIPSSKDPHRCLPGCQICQQLVRCFCGRLVKQHACFTASLAMKYSDVKLGDHFNQAIEEWSVEKHTEQSPTDAYGVINFQGGSHSYRAKYVRLSYDTKPEVILQLLLKEWQMELPKLVISVHGGMQKFELHPRIKQLLGKGLIKAAVTTGAWILTGGVNTGVAKHVGDALKEHASRSSRKICTIGIAPWGVIENRNDLVGRDVVAPYQTLLNPLSKLNVLNNLHSHFILVDDGTVGKYGAEVRLRRELEKTINQQRIHARIGQGVPVVALIFEGGPNVILTVLEYLQESPPVPVVVCEGTGRAADLLAYIHKQTEEGGNLPDAAEPDIISTIKKTFNFGQNEALHLFQTLMECMKRKELITVFHIGSDEHQDIDVAILTALLKGTNASAFDQLILTLAWDRVDIAKNHVFVYGQQWLVGSLEQAMLDALVMDRVAFVKLLIENGVSMHKFLTIPRLEELYNTKQGPTNPMLFHLVRDVKQGNLPPGYKITLIDIGLVIEYLMGGTYRCTYTRKRFRLIYNSLGGNNRRSGRNTSSSTPQLRKSHESFGNRADKKEKMRHNHFIKTAQPYRPKIDTVMEEGKKKRTKDEIVDIDDPETKRFPYPLNELLIWACLMKRQVMARFLWQHGEESMAKALVACKIYRSMAYEAKQSDLVDDTSEELKQYSNDFGQLAVELLEQSFRQDETMAMKLLTYELKNWSNSTCLKLAVSSRLRPFVAHTCTQMLLSDMWMGRLNMRKNSWYKVILSILVPPAILLLEYKTKAEMSHIPQSQDAHQMTMDDSENNFQNITEEIPMEVFKEVRILDSNEGKNEMEIQMKSKKLPITRKFYAFYHAPIVKFWFNTLAYLGFLMLYTFVVLVQMEQLPSVQEWIVIAYIFTYAIEKVREIFMSEAGKVNQKIKVWFSDYFNISDTIAIISFFIGFGLRFGAKWNFANAYDNHVFVAGRLIYCLNIIFWYVRLLDFLAVNQQAGPYVMMIGKMVANMFYIVVIMALVLLSFGVPRKAILYPHEAPSWTLAKDIVFHPYWMIFGEVYAYEIDVCANDSVIPQICGPGTWLTPFLQAVYLFVQYIIMVNLLIAFFNNVYLQVKAISNIVWKYQRYHFIMAYHEKPVLPPPLIILSHIVSLFCCICKRRKKDKTSDGPKLFLTEEDQKKLHDFEEQCVEMYFNEKDDKFHSGSEERIRVTFERVEQMCIQIKEVGDRVNYIKRSLQSLDSQIGHLQDLSALTVDTLKTLTAQKASEASKVHNEITRELSISKHLAQNLIDDGPVRPSVWKKHGVVNTLSSSLPQGDLESNNPFHCNILMKDDKDPQCNIFGQDLPAVPQRKEFNFPEAGSSSGALFPSAVSPPELRQRLHGVELLKIFNKNQKLGSSSTSIPHLSSPPTKFFVSTPSQPSCKSHLETGTKDQETVCSKATEGDNTEFGAFVGHRDSMDLQRFKETSNKIKILSNNNTSENTLKRVSSLAGFTDCHRTSIPVHSKQAEKISRRPSTEDTHEVDSKAALIPDWLQDRPSNREMPSEEGTLNGLTSPFKPAMDTNYYYSAVERNNLMRLSQSIPFTPVPPRGEPVTVYRLEESSPNILNNSMSSWSQLGLCAKIEFLSKEEMGGGLRRAVKVQCTWSEHDILKSGHLYIIKSFLPEVVNTWSSIYKEDTVLHLCLREIQQQRAAQKLTFAFNQMKPKSIPYSPRFLEVFLLYCHSAGQWFAVEECMTGEFRKYNNNNGDEIIPTNTLEEIMLAFSHWTYEYTRGELLVLDLQGVGENLTDPSVIKAEEKRSCDMVFGPANLGEDAIKNFRAKHHCNSCCRKLKLPDLKRNDYTPDKIIFPQDEPSDLNLQPGNSTKESESTNSVRLML",
# FAK2_HUMAN
"MSGVSEPLSRVKLGTLRRPEGPAEPMVVVPVDVEKEDVRILKVCFYSNSFNPGKNFKLVKCTVQTEIREIITSILLSGRIGPNIRLAECYGLRLKHMKSDEIHWLHPQMTVGEVQDKYECLHVEAEWRYDLQIRYLPEDFMESLKEDRTTLLYFYQQLRNDYMQRYASKVSEGMALQLGCLELRRFFKDMPHNALDKKSNFELLEKEVGLDLFFPKQMQENLKPKQFRKMIQQTFQQYASLREEECVMKFFNTLAGFANIDQETYRCELIQGWNITVDLVIGPKGIRQLTSQDAKPTCLAEFKQIRSIRCLPLEEGQAVLQLGIEGAPQALSIKTSSLAEAENMADLIDGYCRLQGEHQGSLIIHPRKDGEKRNSLPQIPMLNLEARRSHLSESCSIESDIYAEIPDETLRRPGGPQYGIAREDVVLNRILGEGFFGEVYEGVYTNHKGEKINVAVKTCKKDCTLDNKEKFMSEAVIMKNLDHPHIVKLIGIIEEEPTWIIMELYPYGELGHYLERNKNSLKVLTLVLYSLQICKAMAYLESINCVHRDIAVRNILVASPECVKLGDFGLSRYIEDEDYYKASVTRLPIKWMSPESINFRRFTTASDVWMFAVCMWEILSFGKQPFFWLENKDVIGVLEKGDRLPKPDLCPPVLYTLMTRCWDYDPSDRPRFTELVCSLSDVYQMEKDIAMEQERNARYRTPKILEPTAFQEPPPKPSRPKYRPPPQTNLLAPKLQFQVPEGLCASSPTLTSPMEYPSPVNSLHTPPLHRHNVFKRHSMREEDFIQPSSREEAQQLWEAEKVKMRQILDKQQKQMVEDYQWLRQEEKSLDPMVYMNDKSPLTPEKEVGYLEFTGPPQKPPRLGAQSIQPTANLDRTDDLVYLNVMELVRAVLELKNELCQLPPEGYVVVVKNVGLTLRKLIGSVDDLLPSLPSSSRTEIEGTQKLLNKDLAELINKMRLAQQNAVTSLSEECKRQMLTASHTLAVDAKNLLDAVDQAKVLANLAHPPAE",
# SRMS_HUMAN
"MEPFLRRRLAFLSFFWDKIWPAGGEPDHGTPGSLDPNTDPVPTLPAEPCSPFPQLFLALYDFTARCGGELSVRRGDRLCALEEGGGYIFARRLSGQPSAGLVPITHVAKASPETLSDQPWYFSGVSRTQAQQLLLSPPNEPGAFLIRPSESSLGGYSLSVRAQAKVCHYRVSMAADGSLYLQKGRLFPGLEELLTYYKANWKLIQNPLLQPCMPQKAPRQDVWERPHSEFALGRKLGEGYFGEVWEGLWLGSLPVAIKVIKSANMKLTDLAKEIQTLKGLRHERLIRLHAVCSGGEPVYIVTELMRKGNLQAFLGTPEGRALRLPPLLGFACQVAEGMSYLEEQRVVHRDLAARNVLVDDGLACKVADFGLARLLKDDIYSPSSSSKIPVKWTAPEAANYRVFSQKSDVWSFGVLLHEVFTYGQCPYEGMTNHETLQQIMRGYRLPRPAACPAEVYVLMLECWRSSPEERPSFATLREKLHAIHRCHP",
# STK10_HUMAN
"MAFANFRRILRLSTFEKRKSREYEHVRRDLDPNEVWEIVGELGDGAFGKVYKAKNKETGALAAAKVIETKSEEELEDYIVEIEILATCDHPYIVKLLGAYYHDGKLWIMIEFCPGGAVDAIMLELDRGLTEPQIQVVCRQMLEALNFLHSKRIIHRDLKAGNVLMTLEGDIRLADFGVSAKNLKTLQKRDSFIGTPYWMAPEVVMCETMKDTPYDYKADIWSLGITLIEMAQIEPPHHELNPMRVLLKIAKSDPPTLLTPSKWSVEFRDFLKIALDKNPETRPSAAQLLEHPFVSSITSNKALRELVAEAKAEVMEEIEDGRDEGEEEDAVDAASTLENHTQNSSEVSPPSLNADKPLEESPSTPLAPSQSQDSVNEPCSQPSGDRSLQTTSPPVVAPGNENGLAVPVPLRKSRPVSMDARIQVAQEKQVAEQGGDLSPAANRSQKASQSRPNSSALETLGGEKLANGSLEPPAQAAPGPSKRDSDCSSLCTSESMDYGTNLSTDLSLNKEMGSLSIKDPKLYKKTLKRTRKFVVDGVEVSITTSKIISEDEKKDEEMRFLRRQELRELRLLQKEEHRNQTQLSNKHELQLEQMHKRFEQEINAKKKFFDTELENLERQQKQQVEKMEQDHAVRRREEARRIRLEQDRDYTRFQEQLKLMKKEVKNEVEKLPRQQRKESMKQKMEEHTQKKQLLDRDFVAKQKEDLELAMKRLTTDNRREICDKERECLMKKQELLRDREAALWEMEEHQLQERHQLVKQQLKDQYFLQRHELLRKHEKEREQMQRYNQRMIEQLKVRQQQEKARLPKIQRSEGKTRMAMYKKSLHINGGGSAAEQREKIKQFSQQEEKRQKSERLQQQQKHENQMRDMLAQCESNMSELQQLQNEKCHLLVEHETQKLKALDESHNQNLKEWRDKLRPRKKALEEDLNQKKREQEMFFKLSEEAECPNPSTPSKAAKFFPYSSADAS",
# M4K3_HUMAN
"MNPGFDLSRRNPQEDFELIQRIGSGTYGDVYKARNVNTGELAAIKVIKLEPGEDFAVVQQEIIMMKDCKHPNIVAYFGSYLRRDKLWICMEFCGGGSLQDIYHVTGPLSELQIAYVSRETLQGLYYLHSKGKMHRDIKGANILLTDNGHVKLADFGVSAQITATIAKRKSFIGTPYWMAPEVAAVERKGGYNQLCDLWAVGITAIELAELQPPMFDLHPMRALFLMTKSNFQPPKLKDKMKWSNSFHHFVKMALTKNPKKRPTAEKLLQHPFVTQHLTRSLAIELLDKVNNPDHSTYHDFDDDDPEPLVAVPHRIHSTSRNVREEKTRSEITFGQVKFDPPLRKETEPHHELPDSDGFLDSSEEIYYTARSNLDLQLEYGQGHQGGYFLGANKSLLKSVEEELHQRGHVAHLEDDEGDDDESKHSTLKAKIPPPLPPKPKSIFIPQEMHSTEDENQGTIKRCPMSGSPAKPSQVPPRPPPPRLPPHKPVALGNGMSSFQLNGERDGSLCQQQNEHRGTNLSRKEKKDVPKPISNGLPPTPKVHMGACFSKVFNGCPLKIHCASSWINPDTRDQYLIFGAEEGIYTLNLNELHETSMEQLFPRRCTWLYVMNNCLLSISGKASQLYSHNLPGLFDYARQMQKLPVAIPAHKLPDRILPRKFSVSAKIPETKWCQKCCVVRNPYTGHKYLCGALQTSIVLLEWVEPMQKFMLIKHIDFPIPCPLRMFEMLVVPEQEYPLVCVGVSRGRDFNQVVRFETVNPNSTSSWFTESDTPQTNVTHVTQLERDTILVCLDCCIKIVNLQGRLKSSRKLSSELTFDFQIESIVCLQDSVLAFWKHGMQGRSFRSNEVTQEISDSTRIFRLLGSDRVVVLESRPTDNPTANSNLYILAGHENSY",
# OXSR1_HUMAN
"MSEDSSALPWSINRDDYELQEVIGSGATAVVQAAYCAPKKEKVAIKRINLEKCQTSMDELLKEIQAMSQCHHPNIVSYYTSFVVKDELWLVMKLLSGGSVLDIIKHIVAKGEHKSGVLDESTIATILREVLEGLEYLHKNGQIHRDVKAGNILLGEDGSVQIADFGVSAFLATGGDITRNKVRKTFVGTPCWMAPEVMEQVRGYDFKADIWSFGITAIELATGAAPYHKYPPMKVLMLTLQNDPPSLETGVQDKEMLKKYGKSFRKMISLCLQKDPEKRPTAAELLRHKFFQKAKNKEFLQEKTLQRAPTISERAKKVRRVPGSSGRLHKTEDGGWEWSDDEFDEESEEGKAAISQLRSPRVKESISNSELFPTTDPVGTLLQVPEQISAHLPQPAGQIATQPTQVSLPPTAEPAKTAQALSSGSGSQETKIPISLVLRLRNSKKELNDIRFEFTPGRDTAEGVSQELISAGLVDGRDLVIVAANLQKIVEEPQSNRSVTFKLASGVEGSDIPDDGKLIGFAQLSIS",
# PAK6_HUMAN
"MFRKKKKKRPEISAPQNFQHRVHTSFDPKEGKFVGLPPQWQNILDTLRRPKPVVDPSRITRVQLQPMKTVVRGSAMPVDGYISGLLNDIQKLSVISSNTLRGRSPTSRRRAQSLGLLGDEHWATDPDMYLQSPQSERTDPHGLYLSCNGGTPAGHKQMPWPEPQSPRVLPNGLAAKAQSLGPAEFQGASQRCLQLGACLQSSPPGASPPTGTNRHGMKAAKHGSEEARPQSCLVGSATGRPGGEGSPSPKTRESSLKRRLFRSMFLSTAATAPPSSSKPGPPPQSKPNSSFRPPQKDNPPSLVAKAQSLPSDQPVGTFSPLTTSDTSSPQKSLRTAPATGQLPGRSSPAGSPRTWHAQISTSNLYLPQDPTVAKGALAGEDTGVVTHEQFKAALRMVVDQGDPRLLLDSYVKIGEGSTGIVCLAREKHSGRQVAVKMMDLRKQQRRELLFNEVVIMRDYQHFNVVEMYKSYLVGEELWVLMEFLQGGALTDIVSQVRLNEEQIATVCEAVLQALAYLHAQGVIHRDIKSDSILLTLDGRVKLSDFGFCAQISKDVPKRKSLVGTPYWMAPEVISRSLYATEVDIWSLGIMVIEMVDGEPPYFSDSPVQAMKRLRDSPPPKLKNSHKVSPVLRDFLERMLVRDPQERATAQELLDHPFLLQTGLPECLVPLIQLYRKQTSTC",
# PAK4_HUMAN
"MFGKRKKRVEISAPSNFEHRVHTGFDQHEQKFTGLPRQWQSLIEESARRPKPLVDPACITSIQPGAPKTIVRGSKGAKDGALTLLLDEFENMSVTRSNSLRRDSPPPPARARQENGMPEEPATTARGGPGKAGSRGRFAGHSEAGGGSGDRRRAGPEKRPKSSREGSGGPQESSRDKRPLSGPDVGTPQPAGLASGAKLAAGRPFNTYPRADTDHPSRGAQGEPHDVAPNGPSAGGLAIPQSSSSSSRPPTRARGAPSPGVLGPHASEPQLAPPACTPAAPAVPGPPGPRSPQREPQRVSHEQFRAALQLVVDPGDPRSYLDNFIKIGEGSTGIVCIATVRSSGKLVAVKKMDLRKQQRRELLFNEVVIMRDYQHENVVEMYNSYLVGDELWVVMEFLEGGALTDIVTHTRMNEEQIAAVCLAVLQALSVLHAQGVIHRDIKSDSILLTHDGRVKLSDFGFCAQVSKEVPRRKSLVGTPYWMAPELISRLPYGPEVDIWSLGIMVIEMVDGEPPYFNEPPLKAMKMIRDNLPPRLKNLHKVSPSLKGFLDRLLVRDPAQRATAAELLKHPFLAKAGPPASIVPLMRQNRTR",
# STK26_HUMAN
"MAHSPVAVQVPGMQNNIADPEELFTKLERIGKGSFGEVFKGIDNRTQQVVAIKIIDLEEAEDEIEDIQQEITVLSQCDSSYVTKYYGSYLKGSKLWIIMEYLGGGSALDLLRAGPFDEFQIATMLKEILKGLDYLHSEKKIHRDIKAANVLLSEQGDVKLADFGVAGQLTDTQIKRNTFVGTPFWMAPEVIQQSAYDSKADIWSLGITAIELAKGEPPNSDMHPMRVLFLIPKNNPPTLVGDFTKSFKEFIDACLNKDPSFRPTAKELLKHKFIVKNSKKTSYLTELIDRFKRWKAEGHSDDESDSEGSDSESTSRENNTHPEWSFTTVRKKPDPKKVQNGAEQDLVQTLSCLSMIITPAFAELKQQDENNASRNQAIEELEKSIAVAEAACPGITDKMVKKLIEKFQKCSADESP",
# STK39_HUMAN
"MAEPSGSPVHVQLPQQAAPVTAAAAAAPAAATAAPAPAAPAAPAPAPAPAAQAVGWPICRDAYELQEVIGSGATAVVQAALCKPRQERVAIKRINLEKCQTSMDELLKEIQAMSQCSHPNVVTYYTSFVVKDELWLVMKLLSGGSMLDIIKYIVNRGEHKNGVLEEAIIATILKEVLEGLDYLHRNGQIHRDLKAGNILLGEDGSVQIADFGVSAFLATGGDVTRNKVRKTFVGTPCWMAPEVMEQVRGYDFKADMWSFGITAIELATGAAPYHKYPPMKVLMLTLQNDPPTLETGVEDKEMMKKYGKSFRKLLSLCLQKDPSKRPTAAELLKCKFFQKAKNREYLIEKLLTRTPDIAQRAKKVRRVPGSSGHLHKTEDGDWEWSDDEMDEKSEEGKAAFSQEKSRRVKEENPEIAVSASTIPEQIQSLSVHDSQGPPNANEDYREASSCAVNLVLRLRNSRKELNDIRFEFTPGRDTADGVSQELFSAGLVDGHDVVIVAANLQKIVDDPKALKTLTFKLASGCDGSEIPDEVKLIGFAQLSVS",
# STRAA_HUMAN
"MSFLVSKPERIRRWVSEKFIVEGLRDLELFGEQPPGDTRRKTNDASSESIASFSKQEVMSSFLPEGGCYELLTVIGKGFEDLMTVNLARYKPTGEYVTVRRINLEACSNEMVTFLQGELHVSKLFNHPNIVPYRATFIADNELWVVTSFMAYGSAKDLICTHFMDGMNELAIAYILQGVLKALDYIHHMGYVHRSVKASHILISVDGKVYLSGLRSNLSMISHGQRQRVVHDFPKYSVKVLPWLSPEVLQQNLQGYDAKSDIYSVGITACELANGHVPFKDMPATQMLLEKLNGTVPCLLDTSTIPAEELTMSPSRSVANSGLSDSLTTSTPRPSNGDSPSHPYHRTFSPHFHHFVEQCLQRNPDARPSASTLLNHSFFKQIKRRASEALPELLRPVTPITNFEGSQSQDHSGIFGLVTNLEELEVDDWEF",
# STRAB_HUMAN
"MSLLDCFCTSRTQVESLRPEKQSETSIHQYLVDEPTLSWSRPSTRASEVLCSTNVSHYELQVEIGRGFDNLTSVHLARHTPTGTLVTIKITNLENCNEERLKALQKAVILSHFFRHPNITTYWTVFTVGSWLWVISPFMAYGSASQLLRTYFPEGMSETLIRNILFGAVRGLNYLHQNGCIHRSIKASHILISGDGLVTLSGLSHLHSLVKHGQRHRAVYDFPQFSTSVQPWLSPELLRQDLHGYNVKSDIYSVGITACELASGQVPFQDMHRTQMLLQKLKGPPYSPLDISIFPQSESRMKNSQSGVDSGIGESVLVSSGTHTVNSDRLHTPSSKTFSPAFFSLVQLCLQQDPEKRPSASSLLSHVFFKQMKEESQDSILSLLPPAYNKPSISLPPVLPWTEPECDFPDEKDSYWEF",
# TAOK3_HUMAN
"MRKGVLKDPEIADLFYKDDPEELFIGLHEIGHGSFGAVYFATNAHTSEVVAIKKMSYSGKQTHEKWQDILKEVKFLRQLKHPNTIEYKGCYLKEHTAWLVMEYCLGSASDLLEVHKKPLQEVEIAAITHGALHGLAYLHSHALIHRDIKAGNILLTEPGQVKLADFGSASMASPANSFVGTPYWMAPEVILAMDEGQYDGKVDIWSLGITCIELAERKPPLFNMNAMSALYHIAQNDSPTLQSNEWTDSFRRFVDYCLQKIPQERPTSAELLRHDFVRRDRPLRVLIDLIQRTKDAVRELDNLQYRKMKKILFQETRNGPLNESQEDEEDSEHGTSLNREMDSLGSNHSIPSMSVSTGSQSSSVNSMQEVMDESSSELVMMHDDESTINSSSSVVHKKDHVFIRDEAGHGDPRPEPRPTQSVQSQALHYRNRERFATIKSASLVTRQIHEHEQENELREQMSGYKRMRRQHQKQLIALENKLKAEMDEHRLKLQKEVETHANNSSIELEKLAKKQVAIIEKEAKVAAADEKKFQQQILAQQKKDLTTFLESQKKQYKICKEKIKEEMNEDHSTPKKEKQERISKHKENLQHTQAEEEAHLLTQQRLYYDKNCRFFKRKIMIKRHEVEQQNIREELNKKRTQKEMEHAMLIRHDESTRELEYRQLHTLQKLRMDLIRLQHQTELENQLEYNKRRERELHRKHVMELRQQPKNLKAMEMQIKKQFQDTCKVQTKQYKALKNHQLEVTPKNEHKTILKTLKDEQTRKLAILAEQYEQSINEMMASQALRLDEAQEAECQALRLQLQQEMELLNAYQSKIKMQTEAQHERELQKLEQRVSLRRAHLEQKIEEELAALQKERSERIKNLLERQEREIETFDMESLRMGFGNLVTLDFPKEDYR",
# TAOK1_HUMAN
"MPSTNRAGSLKDPEIAELFFKEDPEKLFTDLREIGHGSFGAVYFARDVRTNEVVAIKKMSYSGKQSTEKWQDIIKEVKFLQRIKHPNSIEYKGCYLREHTAWLVMEYCLGSASDLLEVHKKPLQEVEIAAITHGALQGLAYLHSHTMIHRDIKAGNILLTEPGQVKLADFGSASMASPANSFVGTPYWMAPEVILAMDEGQYDGKVDVWSLGITCIELAERKPPLFNMNAMSALYHIAQNESPTLQSNEWSDYFRNFVDSCLQKIPQDRPTSEELLKHIFVLRERPETVLIDLIQRTKDAVRELDNLQYRKMKKLLFQEAHNGPAVEAQEEEEEQDHGVGRTGTVNSVGSNQSIPSMSISASSQSSSVNSLPDVSDDKSELDMMEGDHTVMSNSSVIHLKPEEENYREEGDPRTRASDPQSPPQVSRHKSHYRNREHFATIRTASLVTRQMQEHEQDSELREQMSGYKRMRRQHQKQLMTLENKLKAEMDEHRLRLDKDLETQRNNFAAEMEKLIKKHQAAMEKEAKVMSNEEKKFQQHIQAQQKKELNSFLESQKREYKLRKEQLKEELNENQSTPKKEKQEWLSKQKENIQHFQAEEEANLLRRQRQYLELECRRFKRRMLLGRHNLEQDLVREELNKRQTQKDLEHAMLLRQHESMQELEFRHLNTIQKMRCELIRLQHQTELTNQLEYNKRRERELRRKHVMEVRQQPKSLKSKELQIKKQFQDTCKIQTRQYKALRNHLLETTPKSEHKAVLKRLKEEQTRKLAILAEQYDHSINEMLSTQALRLDEAQEAECQVLKMQLQQELELLNAYQSKIKMQAEAQHDRELRELEQRVSLRRALLEQKIEEEMLALQNERTERIRSLLERQAREIEAFDSESMRLGFSNMVLSNLSPEAFSHSYPGASGWSHNPTGGPGPHWGHPMGGPPQAWGHPMQGGPQPWGHPSGPMQGVPRGSSMGVRNSPQALRRTASGGRTEQGMSRSTSVTSQISNGSHMSYT",
# M4K4_HUMAN
"MANDSPAKSLVDIDLSSLRDPAGIFELVEVVGNGTYGQVYKGRHVKTGQLAAIKVMDVTEDEEEEIKLEINMLKKYSHHRNIATYYGAFIKKSPPGHDDQLWLVMEFCGAGSITDLVKNTKGNTLKEDWIAYISREILRGLAHLHIHHVIHRDIKGQNVLLTENAEVKLVDFGVSAQLDRTVGRRNTFIGTPYWMAPEVIACDENPDATYDYRSDLWSCGITAIEMAEGAPPLCDMHPMRALFLIPRNPPPRLKSKKWSKKFFSFIEGCLVKNYMQRPSTEQLLKHPFIRDQPNERQVRIQLKDHIDRTRKKRGEKDETEYEYSGSEEEEEEVPEQEGEPSSIVNVPGESTLRRDFLRLQQENKERSEALRRQQLLQEQQLREQEEYKRQLLAERQKRIEQQKEQRRRLEEQQRREREARRQQEREQRRREQEEKRRLEELERRRKEEEERRRAEEEKRRVEREQEYIRRQLEEEQRHLEVLQQQLLQEQAMLLECRWREMEEHRQAERLQRQLQQEQAYLLSLQHDHRRPHPQHSQQPPPPQQERSKPSFHAPEPKAHYEPADRAREVEDRFRKTNHSSPEAQSKQTGRVLEPPVPSRSESFSNGNSESVHPALQRPAEPQVPVRTTSRSPVLSRRDSPLQGSGQQNSQAGQRNSTSIEPRLLWERVEKLVPRPGSGSSSGSSNSGSQPGSHPGSQSGSGERFRVRSSSKSEGSPSQRLENAVKKPEDKKEVFRPLKPADLTALAKELRAVEDVRPPHKVTDYSSSSEESGTTDEEDDDVEQEGADESTSGPEDTRAASSLNLSNGETESVKTMIVHDDVESEPAMTPSKEGTLIVRQTQSASSTLQKHKSSSSFTPFIDPRLLQISPSSGTTVTSVVGFSCDGMRPEAIRQDPTRKGSVVNVNPTNTRPQSDTPEIRKYKKRFNSEILCAALWGVNLLVGTESGLMLLDRSGQGKVYPLINRRRFQQMDVLEGLNVLVTISGKKDKLRVYYLSWLRNKILHNDPEVEKKQGWTTVGDLEGCVHYKVVKYERIKFLVIALKSSVEVYAWAPKPYHKFMAFKSFGELVHKPLLVDLTVEEGQRLKVIYGSCAGFHAVDVDSGSVYDIYLPTHIQCSIKPHAIIILPNTDGMELLVCYEDEGVYVNTYGRITKDVVLQWGEMPTSVAYIRSNQTMGWGEKAIEIRSVETGHLDGVFMHKRAQRLKFLCERNDKVFFASVRSGGSSQVYFMTLGRTSLLSW",
# TNIK_HUMAN
"MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVKTGQLAAIKVMDVTGDEEEEIKQEINMLKKYSHHRNIATYYGAFIKKNPPGMDDQLWLVMEFCGAGSVTDLIKNTKGNTLKEEWIAYICREILRGLSHLHQHKVIHRDIKGQNVLLTENAEVKLVDFGVSAQLDRTVGRRNTFIGTPYWMAPEVIACDENPDATYDFKSDLWSLGITAIEMAEGAPPLCDMHPMRALFLIPRNPAPRLKSKKWSKKFQSFIESCLVKNHSQRPATEQLMKHPFIRDQPNERQVRIQLKDHIDRTKKKRGEKDETEYEYSGSEEEEEENDSGEPSSILNLPGESTLRRDFLRLQLANKERSEALRRQQLEQQQRENEEHKRQLLAERQKRIEEQKEQRRRLEEQQRREKELRKQQEREQRRHYEEQMRREEERRRAEHEQEYIRRQLEEEQRQLEILQQQLLHEQALLLEYKRKQLEEQRQAERLQRQLKQERDYLVSLQHQRQEQRPVEKKPLYHYKEGMSPSEKPAWAKEVEERSRLNRQSSPAMPHKVANRISDPNLPPRSESFSISGVQPARTPPMLRPVDPQIPHLVAVKSQGPALTASQSVHEQPTKGLSGFQEALNVTSHRVEMPRQNSDPTSENPPLPTRIEKFDRSSWLRQEEDIPPKVPQRTTSISPALARKNSPGNGSALGPRLGSQPIRASNPDLRRTEPILESPLQRTSSGSSSSSSTPSSQPSSQGGSQPGSQAGSSERTRVRANSKSEGSPVLPHEPAKVKPEESRDITRPSRPASYKKAIDEDLTALAKELRELRIEETNRPMKKVTDYSSSSEESESSEEEEEDGESETHDGTVAVSDIPRLIPTGAPGSNEQYNVGMVGTHGLETSHADSFSGSISREGTLMIRETSGEKKRSGHSDSNGFAGHINLPDLVQQSHSPAGTPTEGLGRVSTHSQEMDSGTEYGMGSSTKASFTPFVDPRVYQTSPTDEDEEDEESSAAALFTSELLRQEQAKLNEARKISVVNVNPTNIRPHSDTPEIRKYKKRFNSEILCAALWGVNLLVGTENGLMLLDRSGQGKVYNLINRRRFQQMDVLEGLNVLVTISGKKNKLRVYYLSWLRNRILHNDPEVEKKQGWITVGDLEGCIHYKVVKYERIKFLVIALKNAVEIYAWAPKPYHKFMAFKSFADLQHKPLLVDLTVEEGQRLKVIFGSHTGFHVIDVDSGNSYDIYIPSHIQGNITPHAIVILPKTDGMEMLVCYEDEGVYVNTYGRITKDVVLQWGEMPTSVAYIHSNQIMGWGEKAIEIRSVETGHLDGVFMHKRAQRLKFLCERNDKVFFASVRSGGSSQVFFMTLNRNSMMNW",
# MINK1_HUMAN
"MGDPAPARSLDDIDLSALRDPAGIFELVEVVGNGTYGQVYKGRHVKTGQLAAIKVMDVTEDEEEEIKQEINMLKKYSHHRNIATYYGAFIKKSPPGNDDQLWLVMEFCGAGSVTDLVKNTKGNALKEDCIAYICREILRGLAHLHAHKVIHRDIKGQNVLLTENAEVKLVDFGVSAQLDRTVGRRNTFIGTPYWMAPEVIACDENPDATYDYRSDIWSLGITAIEMAEGAPPLCDMHPMRALFLIPRNPPPRLKSKKWSKKFIDFIDTCLIKTYLSRPPTEQLLKFPFIRDQPTERQVRIQLKDHIDRSRKKRGEKEETEYEYSGSEEEDDSHGEEGEPSSIMNVPGESTLRREFLRLQQENKSNSEALKQQQQLQQQQQRDPEAHIKHLLHQRQRRIEEQKEERRRVEEQQRREREQRKLQEKEQQRRLEDMQALRREEERRQAEREQEYKRKQLEEQRQSERLQRQLQQEHAYLKSLQQQQQQQQLQKQQQQQLLPGDRKPLYHYGRGMNPADKPAWAREVEERTRMNKQQNSPLAKSKPGSTGPEPPIPQASPGPPGPLSQTPPMQRPVEPQEGPHKSLVAHRVPLKPYAAPVPRSQSLQDQPTRNLAAFPASHDPDPAIPAPTATPSARGAVIRQNSDPTSEGPGPSPNPPAWVRPDNEAPPKVPQRTSSIATALNTSGAGGSRPAQAVRARPRSNSAWQIYLQRRAERGTPKPPGPPAQPPGPPNASSNPDLRRSDPGWERSDSVLPASHGHLPQAGSLERNRVGVSSKPDSSPVLSPGNKAKPDDHRSRPGRPADFVLLKERTLDEAPRPPKKAMDYSSSSEEVESSEDDEEEGEGGPAEGSRDTPGGRSDGDTDSVSTMVVHDVEEITGTQPPYGGGTMVVQRTPEEERNLLHADSNGYTNLPDVVQPSHSPTENSKGQSPPSKDGSGDYQSRGLVKAPGKSSFTMFVDLGIYQPGGSGDSIPITALVGGEGTRLDQLQYDVRKGSVVNVNPTNTRAHSETPEIRKYKKRFNSEILCAALWGVNLLVGTENGLMLLDRSGQGKVYGLIGRRRFQQMDVLEGLNLLITISGKRNKLRVYYLSWLRNKILHNDPEVEKKQGWTTVGDMEGCGHYRVVKYERIKFLVIALKSSVEVYAWAPKPYHKFMAFKSFADLPHRPLLVDLTVEEGQRLKVIYGSSAGFHAVDVDSGNSYDIYIPVHIQSQITPHAIIFLPNTDGMEMLLCYEDEGVYVNTYGRIIKDVVLQWGEMPTSVAYICSNQIMGWGEKAIEIRSVETGHLDGVFMHKRAQRLKFLCERNDKVFFASVRSGGSSQVYFMTLNRNCIMNW",
# NRK_HUMAN
"MAGPGGWRDREVTDLGHLPDPTGIFSLDKTIGLGTYGRIYLGLHEKTGAFTAVKVMNARKTPLPEIGRRVRVNKYQKSVGWRYSDEEEDLRTELNLLRKYSFHKNIVSFYGAFFKLSPPGQRHQLWMVMELCAAGSVTDVVRMTSNQSLKEDWIAYICREILQGLAHLHAHRVIHRDIKGQNVLLTHNAEVKLVDFGVSAQVSRTNGRRNSFIGTPYWMAPEVIDCDEDPRRSYDYRSDVWSVGITAIEMAEGAPPLCNLQPLEALFVILRESAPTVKSSGWSRKFHNFMEKCTIKNFLFRPTSANMLQHPFVRDIKNERHVVESLTRHLTGIIKKRQKKGIPLIFEREEAIKEQYTVRRFRGPSCTHELLRLPTSSRCRPLRVLHGEPSQPRWLPDREEPQVQALQQLQGAARVFMPLQALDSAPKPLKGQAQAPQRLQGAARVFMPLQAQVKAKASKPLQMQIKAPPRLRRAARVLMPLQAQVRAPRLLQVQSQVSKKQQAQTQTSEPQDLDQVPEEFQGQDQVPEQQRQGQAPEQQQRHNQVPEQELEQNQAPEQPEVQEQAAEPAQAETEAEEPESLRVNAQVFLPLLSQDHHVLLPLHLDTQVLIPVEGQTEGSPQAQAWTLEPPQAIGSVQALIEGLSRDLLRAPNSNNSKPLGPLQTLMENLSSNRFYSQPEQAREKKSKVSTLRQALAKRLSPKRFRAKSSWRPEKLELSDLEARRQRRQRRWEDIFNQHEEELRQVDKDKEDESSDNDEVFHSIQAEVQIEPLKPYISNPKKIEVQERSPSVPNNQDHAHHVKFSSSVPQRSLLEQAQKPIDIRQRSSQNRQNWLAASESSSEEESPVTGRRSQSSPPYSTIDQKLLVDIHVPDGFKVGKISPPVYLTNEWVGYNALSEIFRNDWLTPAPVIQPPEEDGDYVELYDASADTDGDDDDESNDTFEDTYDHANGNDDLDNQVDQANDVCKDHDDDNNKFVDDVNNNYYEAPSCPRASYGRDGSCKQDGYDGSRGKEEAYRGYGSHTANRSHGGSAASEDNAAIGDQEEHAANIGSERRGSEGDGGKGVVRTSEESGALGLNGEENCSETDGPGLKRPASQDFEYLQEEPGGGNEASNAIDSGAAPSAPDHESDNKDISESSTQSDFSANHSSPSKGSGMSADANFASAILYAGFVEVPEESPKQPSEVNVNPLYVSPACKKPLIHMYEKEFTSEICCGSLWGVNLLLGTRSNLYLMDRSGKADITKLIRRRPFRQIQVLEPLNLLITISGHKNRLRVYHLTWLRNKILNNDPESKRRQEEMLKTEEACKAIDKLTGCEHFSVLQHEETTYIAIALKSSIHLYAWAPKSFDESTAIKVCIDQSADSEGDYMSYQAYIRILAKIQAADPVNRFKRPDELLHLLKLKVFPTLDHKPVTVDLAIGSEKRLKIFFSSADGYHLIDAESEVMSDVTLPKNPLEIIIPQNIIILPDCLGIGMMLTFNAEALSVEANEQLFKKILEMWKDIPSSIAFECTQRTTGWGQKAIEVRSLQSRVLESELKRRSIKKLRFLCTRGDKLFFTSTLRNHHSRVYFMTLGKLEELQSNYDV",
# LATS1_HUMAN
"MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSDAAKAEHNMSKMSTEDPRQVRNPPKFGTHHKALQEIRNSLLPFANETNSSRSTSEVNPQMLQDLQAAGFDEDMVIQALQKTNNRSIEAAIEFISKMSYQDPRREQMAAAAARPINASMKPGNVQQSVNRKQSWKGSKESLVPQRHGPPLGESVAYHSESPNSQTDVGRPLSGSGISAFVQAHPSNGQRVNPPPPPQVRSVTPPPPPRGQTPPPRGTTPPPPSWEPNSQTKRYSGNMEYVISRISPVPPGAWQEGYPPPPLNTSPMNPPNQGQRGISSVPVGRQPIIMQSSSKFNFPSGRPGMQNGTGQTDFMIHQNVVPAGTVNRQPPPPYPLTAANGQSPSALQTGGSAAPSSYTNGSIPQSMMVPNRNSHNMELYNISVPGLQTNWPQSSSAPAQSSPSSGHEIPTWQPNIPVRSNSFNNPLGNRASHSANSQPSATTVTAITPAPIQQPVKSMRVLKPELQTALAPTHPSWIPQPIQTVQPSPFPEGTASNVTVMPPVAEAPNYQGPPPPYPKHLLHQNPSVPPYESISKPSKEDQPSLPKEDESEKSYENVDSGDKEKKQITTSPITVRKNKKDEERRESRIQSYSPQAFKFFMEQHVENVLKSHQQRLHRKKQLENEMMRVGLSQDAQDQMRKMLCQKESNYIRLKRAKMDKSMFVKIKTLGIGAFGEVCLARKVDTKALYATKTLRKKDVLLRNQVAHVKAERDILAEADNEWVVRLYYSFQDKDNLYFVMDYIPGGDMMSLLIRMGIFPESLARFYIAELTCAVESVHKMGFIHRDIKPDNILIDRDGHIKLTDFGLCTGFRWTHDSKYYQSGDHPRQDSMDFSNEWGDPSSCRCGDRLKPLERRAARQHQRCLAHSLVGTPNYIAPEVLLRTGYTQLCDWWSVGVILFEMLVGQPPFLAQTPLETQMKVINWQTSLHIPPQAKLSPEASDLIIKLCRGPEDRLGKNGADEIKAHPFFKTIDFSSDLRQQSASYIPKITHPTDTSNFDPVDPDKLWSDDNEEENVNDTLNGWYKNGKHPEHAFYEFTFRRFFDDNGYPYNYPKPIEYEYINSQGSEQQSDEDDQNTGSEIKNRDLVYV",
# LATS2_HUMAN
"MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTSLDAKVLGSKDATRQQQQMRATPKFGPYQKALREIRYSLLPFANESGTSAAAEVNRQMLQELVNAGCDQEMAGRALKQTGSRSIEAALEYISKMGYLDPRNEQIVRVIKQTSPGKGLMPTPVTRRPSFEGTGDSFASYHQLSGTPYEGPSFGADGPTALEEMPRPYVDYLFPGVGPHGPGHQHQHPPKGYGASVEAAGAHFPLQGAHYGRPHLLVPGEPLGYGVQRSPSFQSKTPPETGGYASLPTKGQGGPPGAGLAFPPPAAGLYVPHPHHKQAGPAAHQLHVLGSRSQVFASDSPPQSLLTPSRNSLNVDLYELGSTSVQQWPAATLARRDSLQKPGLEAPPRAHVAFRPDCPVPSRTNSFNSHQPRPGPPGKAEPSLPAPNTVTAVTAAHILHPVKSVRVLRPEPQTAVGPSHPAWVPAPAPAPAPAPAPAAEGLDAKEEHALALGGAGAFPLDVEYGGPDRRCPPPPYPKHLLLRSKSEQYDLDSLCAGMEQSLRAGPNEPEGGDKSRKSAKGDKGGKDKKQIQTSPVPVRKNSRDEEKRESRIKSYSPYAFKFFMEQHVENVIKTYQQKVNRRLQLEQEMAKAGLCEAEQEQMRKILYQKESNYNRLKRAKMDKSMFVKIKTLGIGAFGEVCLACKVDTHALYAMKTLRKKDVLNRNQVAHVKAERDILAEADNEWVVKLYYSFQDKDSLYFVMDYIPGGDMMSLLIRMEVFPEHLARFYIAELTLAIESVHKMGFIHRDIKPDNILIDLDGHIKLTDFGLCTGFRWTHNSKYYQKGSHVRQDSMEPSDLWDDVSNCRCGDRLKTLEQRARKQHQRCLAHSLVGTPNYIAPEVLLRKGYTQLCDWWSVGVILFEMLVGQPPFLAPTPTETQLKVINWENTLHIPAQVKLSPEARDLITKLCCSADHRLGRNGADDLKAHPFFSAIDFSSDIRKQPAPYVPTISHPMDTSNFDPVDEESPWNDASEGSTKAWDTLTSPNNKHPEHAFYEFTFRRFFDDNGYPFRCPKPSGAEASQAESSDLESSDLVDQTEGCQPVYV",
# CDK19_HUMAN
"MDYDFKAKLAAERERVEDLFEYEGCKVGRGTYGHVYKARRKDGKDEKEYALKQIEGTGISMSACREIALLRELKHPNVIALQKVFLSHSDRKVWLLFDYAEHDLWHIIKFHRASKANKKPMQLPRSMVKSLLYQILDGIHYLHANWVLHRDLKPANILVMGEGPERGRVKIADMGFARLFNSPLKPLADLDPVVVTFWYRAPELLLGARHYTKAIDIWAIGCIFAELLTSEPIFHCRQEDIKTSNPFHHDQLDRIFSVMGFPADKDWEDIRKMPEYPTLQKDFRRTTYANSSLIKYMEKHKVKPDSKVFLLLQKLLTMDPTKRITSEQALQDPYFQEDPLPTLDVFAGCQIPYPKREFLNEDDPEEKGDKNQQQQQNQHQQPTAPPQQAAAPPQAPPPQQNSTQTNGTAGGAGAGVGGTGAGLQHSQDSSLNQVPPNKKPRLGPSGANSGGPVMPSDYQHSSSRLNYQSSVQGSSQSQSTLGYSSSSQQSSQYHPSHQAHRY",
# NIM1_HUMAN
"MTAVYMNGGGLVNPHYARWDRRDSVESGCQTESSKEGEEGQPRQLTPFEKLTQDMSQDEKVVREITLGKRIGFYRIRGEIGSGNFSQVKLGIHSLTKEKVAIKILDKTKLDQKTQRLLSREISSMEKLHHPNIIRLYEVVETLSKLHLVMEYAGGGELFGKISTEGKLSEPESKLIFSQIVSAVKHMHENQIIHRDLKAENVFYTSNTCVKVGDFGFSTVSKKGEMLNTFCGSPPYAAPELFRDEHYIGIYVDIWALGVLLYFMVTGTMPFRAETVAKLKKSILEGTYSVPPHVSEPCHRLIRGVLQQIPTERYGIDCIMNDEWMQGVPYPTPLEPFQLDPKHLSETSTLKEEENEVKSTLEHLGITEEHIRNNQGRDARSSITGVYRIILHRVQRKKALESVPVMMLPDPKERDLKKGSRVYRGIRHTSKFCSIL",
# ULK3_HUMAN
"MAGPGWGPPRLDGFILTERLGSGTYATVYKAYAKKDTREVVAIKCVAKKSLNKASVENLLTEIEILKGIRHPHIVQLKDFQWDSDNIYLIMEFCAGGDLSRFIHTRRILPEKVARVFMQQLASALQFLHERNISHLDLKPQNILLSSLEKPHLKLADFGFAQHMSPWDEKHVLRGSPLYMAPEMVCQRQYDARVDLWSMGVILYEALFGQPPFASRSFSELEEKIRSNRVIELPLRPLLSRDCRDLLQRLLERDPSRRISFQDFFAHPWVDLEHMPSGESLGRATALVVQAVKKDQEGDSAAALSLYCKALDFFVPALHYEVDAQRKEAIKAKVGQYVSRAEELKAIVSSSNQALLRQGTSARDLLREMARDKPRLLAALEVASAAMAKEEAAGGEQDALDLYQHSLGELLLLLAAEPPGRRRELLHTEVQNLMARAEYLKEQVKMRESRWEADTLDKEGLSESVRSSCTLQ",
# PDK1L_HUMAN
"MVSSQPKYDLIREVGRGSYGVVYEAVIRKTSARVAVKKIRCHAPENVELALREFWALSSIKSQHPNVIHLEECILQKDGMVQKMSHGSNSSLYLQLVETSLKGEIAFDPRSAYYLWFVMDFCDGGDMNEYLLSRKPNRKTNTSFMLQLSSALAFLHKNQIIHRDLKPDNILISQTRLDTSDLEPTLKVADFGLSKVCSASGQNPEEPVSVNKCFLSTACGTDFYMAPEVWEGHYTAKADIFALGIIIWAMLERITFIDTETKKELLGSYVKQGTEIVPVGEALLENPKMELLIPVKKKSMNGRMKQLIKEMLAANPQDRPDAFELELRLVQIAFKDSSWET",
# TTBK2_HUMAN
"MSGGGEQLDILSVGILVKERWKVLRKIGGGGFGEIYDALDMLTRENVALKVESAQQPKQVLKMEVAVLKKLQGKDHVCRFIGCGRNDRFNYVVMQLQGRNLADLRRSQSRGTFTISTTLRLGRQILESIESIHSVGFLHRDIKPSNFAMGRFPSTCRKCYMLDFGLARQFTNSCGDVRPPRAVAGFRGTVRYASINAHRNREMGRHDDLWSLFYMLVEFVVGQLPWRKIKDKEQVGSIKERYDHRLMLKHLPPEFSIFLDHISSLDYFTKPDYQLLTSVFDNSIKTFGVIESDPFDWEKTGNDGSLTTTTTSTTPQLHTRLTPAAIGIANATPIPGDLLRENTDEVFPDEQLSDGENGIPVGVSPDKLPGSLGHPRPQEKDVWEEMDANKNKIKLGICKAATEEENSHGQANGLLNAPSLGSPIRVRSEITQPDRDIPLVRKLRSIHSFELEKRLTLEPKPDTDKFLETCLEKMQKDTSAGKESILPALLHKPCVPAVSRTDHIWHYDEEYLPDASKPASANTPEQADGGGSNGFIAVNLSSCKQEIDSKEWVIVDKEQDLQDFRTNEAVGHKTTGSPSDEEPEVLQVLEASPQDEKLQLGPWAENDHLKKETSGVVLALSAEGPPTAASEQYTDRLELQPGAASQFIAATPTSLMEAQAEGPLTAITIPRPSVASTQSTSGSFHCGQQPEKKDLQPMEPTVELYSPRENFSGLVVTEGEPPSGGSRTDLGLQIDHIGHDMLPNIRESNKSQDLGPKELPDHNRLVVREFENLPGETEEKSILLESDNEDEKLSRGQHCIEISSLPGDLVIVEKDHSATTEPLDVTKTQTFSVVPNQDKNNEIMKLLTVGTSEISSRDIDPHVEGQIGQVAEMQKNKISKDDDIMSEDLPGHQGDLSTFLHQEGKREKITPRNGELFHCVSENEHGAPTRKDMVRSSFVTRHSRIPVLAQEIDSTLESSSPVSAKEKLLQKKAYQPDLVKLLVEKRQFKSFLGDLSSASDKLLEEKLATVPAPFCEEEVLTPFSRLTVDSHLSRSAEDSFLSPIISQSRKSKIPRPVSWVNTDQVNSSTSSQFFPRPPPGKPPTRPGVEARLRRYKVLGSSNSDSDLFSRLAQILQNGSQKPRSTTQCKSPGSPHNPKTPPKSPVVPRRSPSASPRSSSLPRTSSSSPSRAGRPHHDQRSSSPHLGRSKSPPSHSGSSSSRRSCQQEHCKPSKNGLKGSGSLHHHSASTKTPQGKSKPASKLSR",
# SCYL1_HUMAN
"MWFFARDPVRDFPFELIPEPPEGGLPGPWALHRGRKKATGSPVSIFVYDVKPGAEEQTQVAKAAFKRFKTLRHPNILAYIDGLETEKCLHVVTEAVTPLGIYLKARVEAGGLKELEISWGLHQIVKALSFLVNDCSLIHNNVCMAAVFVDRAGEWKLGGLDYMYSAQGNGGGPPRKGIPELEQYDPPELADSSGRVVREKWSADMWRLGCLIWEVFNGPLPRAAALRNPGKIPKTLVPHYCELVGANPKVRPNPARFLQNCRAPGGFMSNRFVETNLFLEEIQIKEPAEKQKFFQELSKSLDAFPEDFCRHKVLPQLLTAFEFGNAGAVVLTPLFKVGKFLSAEEYQQKIIPVVVKMFSSTDRAMRIRLLQQMEQFIQYLDEPTVNTQIFPHVVHGFLDTNPAIREQTVKSMLLLAPKLNEANLNVELMKHFARLQAKDEQGPIRCNTTVCLGKIGSYLSASTRHRVLTSAFSRATRDPFAPSRVAGVLGFAATHNLYSMNDCAQKILPVLCGLTVDPEKSVRDQAFKAIRSFLSKLESVSEDPTQLEEVEKDVHAASSPGMGGAAASWAGWAVTGVSSLTSKLIRSHPTTAPTETNIPQRPTPEGVPAPAPTPVPATPTTSGHWETQEEDKDTAEDSSTADRWDDEDWGSLEQEAESVLAQQDDWSTGGQVSRASQVSNSDHKSSKSPESDWSSWEAEGSWEQGWQEPSSQEPPPDGTRLASEYNWGGPESSDKGDPFATLSARPSTQPRPDSWGEDNWEGLETDSRQVKAELARKKREERRREMEAKRAERKVAKGPMKLGARKLD",
# GWL_HUMAN
"MDPTAGSKKEPGGGAATEEGVNRIAVPKPPSIEEFSIVKPISRGAFGKVYLGQKGGKLYAVKVVKKADMINKNMTHQVQAERDALALSKSPFIVHLYYSLQSANNVYLVMEYLIGGDVKSLLHIYGYFDEEMAVKYISEVALALDYLHRHGIIHRDLKPDNMLISNEGHIKLTDFGLSKVTLNRDINMMDILTTPSMAKPRQDYSRTPGQVLSLISSLGFNTPIAEKNQDPANILSACLSETSQLSQGLVCPMSVDQKDTTPYSSKLLKSCLETVASNPGMPVKCLTSNLLQSRKRLATSSASSQSHTFISSVESECHSSPKWEKDCQESDEALGPTMMSWNAVEKLCAKSANAIETKGFNKKDLELALSPIHNSSALPTTGRSCVNLAKKCFSGEVSWEAVELDVNNINMDTDTSQLGFHQSNQWAVDSGGISEEHLGKRSLKRNFELVDSSPCKKIIQNKKTCVEYKHNEMTNCYTNQNTGLTVEVQDLKLSVHKSQQNDCANKENIVNSFTDKQQTPEKLPIPMIAKNLMCELDEDCEKNSKRDYLSSSFLCSDDDRASKNISMNSDSSFPGISIMESPLESQPLDSDRSIKESSFEESNIEDPLIVTPDCQEKTSPKGVENPAVQESNQKMLGPPLEVLKTLASKRNAVAFRSFNSHINASNNSEPSRMNMTSLDAMDISCAYSGSYPMAITPTQKRRSCMPHQQTPNQIKSGTPYRTPKSVRRGVAPVDDGRILGTPDYLAPELLLGRAHGPAVDWWALGVCLFEFLTGIPPFNDETPQQVFQNILKRDIPWPEGEEKLSDNAQSAVEILLTIDDTKRAGMKELKRHPLFSDVDWENLQHQTMPFIPQPDDETDTSYFEARNTAQHLTVSGFSL",
# PINK1_HUMAN
"MAVRQALGRGLQLGRALLLRFTGKPGRAYGLGRPGPAAGCVRGERPGWAAGPGAEPRRVGLGLPNRLRFFRQSVAGLAARLQRQFVVRAWGCAGPCGRAVFLAFGLGLGLIEEKQAESRRAVSACQEIQAIFTQKSKPGPDPLDTRRLQGFRLEEYLIGQSIGKGCSAAVYEATMPTLPQNLEVTKSTGLLPGRGPGTSAPGEGQERAPGAPAFPLAIKMMWNISAGSSSEAILNTMSQELVPASRVALAGEYGAVTYRKSKRGPKQLAPHPNIIRVLRAFTSSVPLLPGALVDYPDVLPSRLHPEGLGHGRTLFLVMKNYPCTLRQYLCVNTPSPRLAAMMLLQLLEGVDHLVQQGIAHRDLKSDNILVELDPDGCPWLVIADFGCCLADESIGLQLPFSSWYVDRGGNGCLMAPEVSTARPGPRAVIDYSKADAWAVGAIAYEIFGLVNPFYGQGKAHLESRSYQEAQLPALPESVPPDVRQLVRALLQREASKRPSARVAANVLHLSLWGEHILALKNLKLDKMVGWLLQQSAATLLANRLTEKCCVETKMKMLFLANLECETLCQAALLLCSWRAAL",
# ULK4_HUMAN
"MENFILYEEIGRGSKTVVYKGRRKGTINFVAILCTDKCKRPEITNWVRLTREIKHKNIVTFHEWYETSNHLWLVVELCTGGSLKTVIAQDENLPEDVVREFGIDLISGLHHLHKLGILFCDISPRKILLEGPGTLKFSNFCLAKVEGENLEEFFALVAAEEGGGDNGENVLKKSMKSRVKGSPVYTAPEVVRGADFSISSDLWSLGCLLYEMFSGKPPFFSESISELTEKILCEDPLPPIPKDSSRPKASSDFINLLDGLLQRDPQKRLTWTRLLQHSFWKKAFAGADQESSVEDLSLSRNTMECSGPQDSKELLQNSQSRQAKGHKSGQPLGHSFRLENPTEFRPKSTLEGQLNESMFLLSSRPTPRTSTAVEVSPGEDMTHCSPQKTSPLTKITSGHLSQQDLESQMRELIYTDSDLVVTPIIDNPKIMKQPPVKFDAKILHLPTYSVDKLLFLKDQDWNDFLQQVCSQIDSTEKSMGASRAKLNLLCYLCVVAGHQEVATRLLHSPLFQLLIQHLRIAPNWDIRAKVAHVIGLLASHTAELQENTPVVEAIVLLTELIRENFRNSKLKQCLLPTLGELIYLVATQEEKKKNPRECWAVPLAAYTVLMRCLREGEERVVNHMAAKIIENVCTTFSAQSQGFITGEIGPILWYLFRHSTADSLRITAVSALCRITRHSPTAFQNVIEKVGLNSVINSLASAICKVQQYMLTLFAAMLSCGIHLQRLIQEKGFVSTIIRLLDSPSTCIRAKAFLVLLYILIYNREMLLLSCQARLVMYIERDSRKTTPGKEQQSGNEYLSKCLDLLICHIVQELPRILGDILNSLANVSGRKHPSTVQVKQLKLCLPLMPVVLHLVTSQVFRPQVVTEEFLFSYGTILSHIKSVDSGETNIDGAIGLTASEEFIKITLSAFEAIIQYPILLKDYRSTVVDYILPPLVSLVQSQNVEWRLFSLRLLSETTSLLVNQEFGDGKEKASVDSDSNLLALIRDVLLPQYEHILLEPDPVPAYALKLLVAMTEHNPTFTRLVEESKLIPLIFEVTLEHQESILGNTMQSVIALLSNLVACKDSNMELLYEQGLVSHICNLLTETATLCLDVDNKNNNEMAAPLLFSLLDILHSMLTYTSGIVRLALQAQKSGSGEDPQAAEDLLLLNRPLTDLISLLIPLLPNEDPEIFDVSSKCLSILVQLYGGENPDSLSPENVEIFAHLLTSKEDPKEQKLLLRILRRMITSNEKHLESLKNAGSLLRALERLAPGSGSFADSAVAPLALEILQAVGH",
# MLKL_HUMAN
"MENLKHIITLGQVIHKRCEEMKYCKKQCRRLGHRVLGLIKPLEMLQDQGKRSVPSEKLTTAMNRFKAALEEANGEIEKFSNRSNICRFLTASQDKILFKDVNRKLSDVWKELSLLLQVEQRMPVSPISQGASWAQEDQQDADEDRRAFQMLRRDNEKIEASLRRLEINMKEIKETLRQYLPPKCMQEIPQEQIKEIKKEQLSGSPWILLRENEVSTLYKGEYHRAPVAIKVFKKLQAGSIAIVRQTFNKEIKTMKKFESPNILRIFGICIDETVTPPQFSIVMEYCELGTLRELLDREKDLTLGKRMVLVLGAARGLYRLHHSEAPELHGKIRSSNFLVTQGYQVKLAGFELRKTQTSMSLGTTREKTDRVKSTAYLSPQELEDVFYQYDVKSEIYSFGIVLWEIATGDIPFQGCNSEKIRKLVAVKRQQEPLGEDCPSELREIIDECRAHDPSVRPSVDEILKKLSTFSK",
# DCLK3_HUMAN
"MGKEPLTLKSIQVAVEELYPNKARALTLAQHSRAPSPRLRSRLFSKALKGDHRCGETETPKSCSEVAGCKAAMRHQGKIPEELSLDDRARTQKKWGRGKWEPEPSSKPPREATLEERHARGEKHLGVEIEKTSGEIIRCEKCKRERELQQSLERERLSLGTSELDMGKGPMYDVEKLVRTRSCRRSPEANPASGEEGWKGDSHRSSPRNPTQELRRPSKSMDKKEDRGPEDQESHAQGAAKAKKDLVEVLPVTEEGLREVKKDTRPMSRSKHGGWLLREHQAGFEKLRRTRGEEKEAEKEKKPCMSGGRRMTLRDDQPAKLEKEPKTRPEENKPERPSGRKPRPMGIIAANVEKHYETGRVIGDGNFAVVKECRHRETRQAYAMKIIDKSRLKGKEDMVDSEILIIQSLSHPNIVKLHEVYETDMEIYLILEYVQGGDLFDAIIESVKFPEPDAALMIMDLCKALVHMHDKSIVHRDLKPENLLVQRNEDKSTTLKLADFGLAKHVVRPIFTVCGTPTYVAPEILSEKGYGLEVDMWAAGVILYILLCGFPPFRSPERDQDELFNIIQLGHFEFLPPYWDNISDAAKDLVSRLLVVDPKKRYTAHQVLQHPWIETAGKTNTVKRQKQVSPSSEGHFRSQHKRVVEQVS",
# PKDCC_HUMAN
"MRRRRAAVAAGFCASFLLGSVLNVLFAPGSEPPRPGQSPEPSPAPGPGRRGGRGELARQIRARYEEVQRYSRGGPGPGAGRPERRRLMDLAPGGPGLPRPRPPWARPLSDGAPGWPPAPGPGSPGPGPRLGCAALRNVSGAQYMGSGYTKAVYRVRLPGGAAVALKAVDFSGHDLGSCVREFGVRRGCYRLAAHKLLKEMVLLERLRHPNVLQLYGYCYQDSEDIPDTLTTITELGAPVEMIQLLQTSWEDRFRICLSLGRLLHHLAHSPLGSVTLLDFRPRQFVLVDGELKVTDLDDARVEETPCAGSTDCILEFPARNFTLPCSAQGWCEGMNEKRNLYNAYRFFFTYLLPHSAPPSLRPLLDSIVNATGELAWGVDETLAQLEKVLHLYRSGQYLQNSTASSSTEYQCIPDSTIPQEDYRCWPSYHHGSCLLSVFNLAEAVDVCESHAQCRAFVVTNQTTWTGRQLVFFKTGWSQVVPDPNKTTYVKASG",
# CDK15_HUMAN
"MGQELCAKTVQPGCSCYHCSEGGEAHSCRRSQPETTEAAFKLTDLKEASCSMTSFHPRGLQAARAQKFKSKRPRSNSDCFQEEDLRQGFQWRKSLPFGAASSYLNLEKLGEGSYATVYKGISRINGQLVALKVISMNAEEGVPFTAIREASLLKGLKHANIVLLHDIIHTKETLTFVFEYMHTDLAQYMSQHPGGLHPHNVRLFMFQLLRGLAYIHHQHVLHRDLKPQNLLISHLGELKLADFGLARAKSIPSQTYSSEVVTLWYRPPDALLGATEYSSELDIWGAGCIFIEMFQGQPLFPGVSNILEQLEKIWEVLGVPTEDTWPGVSKLPNYNPEWFPLPTPRSLHVVWNRLGRVPEAEDLASQMLKGFPRDRVSAQEALVHDYFSALPSQLYQLPDEESLFTVSGVRLKPEMCDLLASYQKGHHPAQFSKCW",
# STK33_HUMAN
"MADSGLDKKSTKCPDCSSASQKDVLCVCSSKTRVPPVLVVEMSQTSSIGSAESLISLERKKEKNINRDITSRKDLPSRTSNVERKASQQQWGRGNFTEGKVPHIRIENGAAIEEIYTFGRILGKGSFGIVIEATDKETETKWAIKKVNKEKAGSSAVKLLEREVNILKSVKHEHIIHLEQVFETPKKMYLVMELCEDGELKEILDRKGHFSENETRWIIQSLASAIAYLHNNDIVHRDLKLENIMVKSSLIDDNNEINLNIKVTDFGLAVKKQSRSEAMLQATCGTPIYMAPEVISAHDYSQQCDIWSIGVVMYMLLRGEPPFLASSEEKLFELIRKGELHFENAVWNSISDCAKSVLKQLMKVDPAHRITAKELLDNQWLTGNKLSSVRPTNVLEMMKEWKNNPESVEENTTEEKNKPSTEEKLKSYQPWGNVPDANYTSDEEEEKQSTAYEKQFPATSKDNFDMCSSSFTSSKLLPAEIKGEMEKTPVTPSQGTATKYPAKSGALSRTKKKL",
# PRPK_HUMAN
"MAAARATTPADGEEPAPEAEALAAARERSSRFLSGLELVKQGAEARVFRGRFQGRAAVIKHRFPKGYRHPALEARLGRRRTVQEARALLRCRRAGISAPVVFFVDYASNCLYMEEIEGSVTVRDYIQSTMETEKTPQGLSNLAKTIGQVLARMHDEDLIHGDLTTSNMLLKPPLEQLNIVLIDFGLSFISALPEDKGVDLYVLEKAFLSTHPNTETVFEAFLKSYSTSSKKARPVLKKLDEVRLRGRKRSMVG",
# MK15_HUMAN
"MCTVVDPRIVRRYLLRRQLGQGAYGIVWKAVDRRTGEVVAIKKIFDAFRDKTDAQRTFREITLLQEFGDHPNIISLLDVIRAENDRDIYLVFEFMDTDLNAVIRKGGLLQDVHVRSIFYQLLRATRFLHSGHVVHRDQKPSNVLLDANCTVKLCDFGLARSLGDLPEGPEDQAVTEYVATRWYRAPEVLLSSHRYTLGVDMWSLGCILGEMLRGRPLFPGTSTLHQLELILETIPPPSEEDLLALGSGCRASVLHQLGSRPRQTLDALLPPDTSPEALDLLRRLLVFAPDKRLSATQALQHPYVQRFHCPSDEWAREADVRPRAHEGVQLSVPEYRSRVYQMILECGGSSGTSREKGPEGVSPSQAHLHKPRADPQLPSRTPVQGPRPRPQSSPGHDPAEHESPRAAKNVPRQNSAPLLQTALLGNGERPPGAKEAPPLTLSLVKPSGRGAAPSLTSQAAAQVANQALIRGDWNRGGGVRVASVQQVPPRLPPEARPGRRMFSTSALQGAQGGARALLGGYSQAYGTVCHSALGHLPLLEGHHV",
# CDKL4_HUMAN
"MEKYEKLAKTGEGSYGVVFKCRNKTSGQVVAVKKFVESEDDPVVKKIALREIRMLKQLKHPNLVNLIEVFRRKRKMHLVFEYCDHTLLNELERNPNGVADGVIKSVLWQTLQALNFCHIHNCIHRDIKPENILITKQGIIKICDFGFAQILIPGDAYTDYVATRWYRAPELLVGDTQYGSSVDIWAIGCVFAELLTGQPLWPGKSDVDQLYLIIRTLGKLIPRHQSIFKSNGFFHGISIPEPEDMETLEEKFSDVHPVALNFMKGCLKMNPDDRLTCSQLLESSYFDSFQEAQIKRKARNEGRNRRRQQQAPKSAFPRLFLKTKICQVQRNETQTSGNQILPNGPILQNSMVTVMTNINSAVYQVTVLHLLSENFEVKS",
# PACE1_HUMAN
"MGSENSALKSYTLREPPFTLPSGLAVYPAVLQDGKFASVFVYKRENEDKVNKAAKHLKTLRHPCLLRFLSCTVEADGIHLVTERVQPLEVALETLSSAEVCAGIYDILLALIFLHDRGHLTHNNVCLSSVFVSEDGHWKLGGMETVCKVSQATPEFLRSIQSIRDPASIPPEEMSPEFTTLPECHGHARDAFSFGTLVESLLTILNEQVSADVLSSFQQTLHSTLLNPIPKCRPALCTLLSHDFFRNDFLEVVNFLKSLTLKSEEEKTEFFKFLLDRVSCLSEELIASRLVPLLLNQLVFAEPVAVKSFLPYLLGPKKDHAQGETPCLLSPALFQSRVIPVLLQLFEVHEEHVRMVLLSHIEAYVEHFTQEQLKKVILPQVLLGLRDTSDSIVAITLHSLAVLVSLLGPEVVVGGERTKIFKRTAPSFTKNTDLSLEDSPMCVVCSHHSQISPILENPFSSIFPKCFFSGSTPINSKKHIQRDYYNTLLQTGDPFSQPIKFPINGLSDVKNTSEDSENFPSSSKKSEEWPDWSEPEEPENQTVNIQIWPREPCDDVKSQCTTLDVEESSWDDCEPSSLDTKVNPGGGITATKPVTSGEQKPIPALLSLTEESMPWKSSLPQKISLVQRGDDADQIEPPKVSSQERPLKVPSELGLGEEFTIQVKKKPVKDPEMDWFADMIPEIKPSAAFLILPELRTEMVPKKDDVSPVMQFSSKFAAAEITEGEAEGWEEEGELNWEDNNW",
# ST32C_HUMAN
"MRSGAERRGSSAAASPGSPPPGRARPAGSDAPSALPPPAAGQPRARDSGDVRSQPRPLFQWSKWKKRMGSSMSAATARRPVFDDKEDVNFDHFQILRAIGKGSFGKVCIVQKRDTEKMYAMKYMNKQQCIERDEVRNVFRELEILQEIEHVFLVNLWYSFQDEEDMFMVVDLLLGGDLRYHLQQNVQFSEDTVRLYICEMALALDYLRGQHIIHRDVKPDNILLDERGHAHLTDFNIATIIKDGERATALAGTKPYMAPEIFHSFVNGGTGYSFEVDWWSVGVMAYELLRGWRPYDIHSSNAVESLVQLFSTVSVQYVPTWSKEMVALLRKLLTVNPEHRLSSLQDVQAAPALAGVLWDHLSEKRVEPGFVPNKGRLHCDPTFELEEMILESRPLHKKKKRLAKNKSRDNSRDSSQSENDYLQDCLDAIQQDFVIFNREKLKRSQDLPREPLPAPESRDAAEPVEDEAERSALPMCGPICPSAGSG",
# NEK9_HUMAN
"MSVLGEYERHCDSINSDFGSESGGCGDSSPGPSASQGPRAGGGAAEQEELHYIPIRVLGRGAFGEATLYRRTEDDSLVVWKEVDLTRLSEKERRDALNEIVILALLQHDNIIAYYNHFMDNTTLLIELEYCNGGNLYDKILRQKDKLFEEEMVVWYLFQIVSAVSCIHKAGILHRDIKTLNIFLTKANLIKLGDYGLAKKLNSEYSMAETLVGTPYYMSPELCQGVKYNFKSDIWAVGCVIFELLTLKRTFDATNPLNLCVKIVQGIRAMEVDSSQYSLELIQMVHSCLDQDPEQRPTADELLDRPLLRKRRREMEEKVTLLNAPTKRPRSSTVTEAPIAVVTSRTSEVYVWGGGKSTPQKLDVIKSGCSARQVCAGNTHFAVVTVEKELYTWVNMQGGTKLHGQLGHGDKASYRQPKHVEKLQGKAIRQVSCGDDFTVCVTDEGQLYAFGSDYYGCMGVDKVAGPEVLEPMQLNFFLSNPVEQVSCGDNHVVVLTRNKEVYSWGCGEYGRLGLDSEEDYYTPQKVDVPKALIIVAVQCGCDGTFLLTQSGKVLACGLNEFNKLGLNQCMSGIINHEAYHEVPYTTSFTLAKQLSFYKIRTIAPGKTHTAAIDERGRLLTFGCNKCGQLGVGNYKKRLGINLLGGPLGGKQVIRVSCGDEFTIAATDDNHIFAWGNGGNGRLAMTPTERPHGSDICTSWPRPIFGSLHHVPDLSCRGWHTILIVEKVLNSKTIRSNSSGLSIGTVFQSSSPGGGGGGGGGEEEDSQQESETPDPSGGFRGTMEADRGMEGLISPTEAMGNSNGASSSCPGWLRKELENAEFIPMPDSPSPLSAAFSESEKDTLPYEELQGLKVASEAPLEHKPQVEASSPRLNPAVTCAGKGTPLTPPACACSSLQVEVERLQGLVLKCLAEQQKLQQENLQIFTQLQKLNKKLEGGQQVGMHSKGTQTAKEEMEMDPKPDLDSDSWCLLGTDSCRPSL",
# TSSK3_HUMAN
"MEDFLLSNGYQLGKTIGEGTYSKVKEAFSKKHQRKVAIKVIDKMGGPEEFIQRFLPRELQIVRTLDHKNIIQVYEMLESADGKICLVMELAEGGDVFDCVLNGGPLPESRAKALFRQMVEAIRYCHGCGVAHRDLKCENALLQGFNLKLTDFGFAKVLPKSHRELSQTFCGSTAYAAPEVLQGIPHDSKKGDVWSMGVVLYVMLCASLPFDDTDIPKMLWQQQKGVSFPTHLSISADCQDLLKRLLEPDMILRPSIEEVSWHPWLAST",
# NUAK2_HUMAN
"MESLVFARRSGPTPSAAELARPLAEGLIKSPKPLMKKQAVKRHHHKHNLRHRYEFLETLGKGTYGKVKKARESSGRLVAIKSIRKDKIKDEQDLMHIRREIEIMSSLNHPHIIAIHEVFENSSKIVIVMEYASRGDLYDYISERQQLSEREARHFFRQIVSAVHYCHQNRVVHRDLKLENILLDANGNIKIADFGLSNLYHQGKFLQTFCGSPLYASPEIVNGKPYTGPEVDSWSLGVLLYILVHGTMPFDGHDHKILVKQISNGAYREPPKPSDACGLIRWLLMVNPTRRATLEDVASHWWVNWGYATRVGEQEAPHEGGHPGSDSARASMADWLRRSSRPLLENGAKVCSFFKQHAPGGGSTTPGLERQHSLKKSRKENDMAQSLHSDTADDTAHRPGKSNLKLPKGILKKKVSASAEGVQEDPPELSPIPASPGQAAPLLPKKGILKKPRQRESGYYSSPEPSESGELLDAGDVFVSGDPKEQKPPQASGLLLHRKGILKLNGKFSQTALELAAPTTFGSLDELAPPRPLARASRPSGAVSEDSILSSESFDQLDLPERLPEPPLRGCVSVDNLTGLEEPPSEGPGSCLRRWRQDPLGDSCFSLTDCQEVTATYRQALRVCSKLT",
# RPKL1_HUMAN
"MSLVACECLPSPGLEPEPCSRARSQAHVYLEQIRNRVALGVPDMTKRDYLVDAATQIRLALERDVSEDYEAAFNHYQNGVDVLLRGIHVDPNKERREAVKLKITKYLRRAEEIFNCHLQRPLSSGASPSAGFSSLRLRPIRTLSSAVEQLRGCRVVGVIEKVQLVQDPATGGTFVVKSLPRCHMVSRERLTIIPHGVPYMTKLLRYFVSEDSIFLHLEHVQGGTLWSHLLSQAHSRHSGLSSGSTQERMKAQLNPHLNLLTPARLPSGHAPGQDRIALEPPRTSPNLLLAGEAPSTRPQREAEGEPTARTSTSGSSDLPKAPGGHLHLQARRAGQNSDAGPPRGLTWVPEGAGPVLGGCGRGMDQSCLSADGAGRGCGRATWSVREEQVKQWAAEMLVALEALHEQGVLCRDLHPGNLLLDQAGHIRLTYFGQWSEVEPQCCGEAVDNLYSAPEVGGISELTEACDWWSFGSLLYELLTGMALSQSHPSGIQAHTQLQLPEWLSRPAASLLTELLQFEPTRRLGMGEGGVSKLKSHPFFSTIQWSKLVG",
# TSSK2_HUMAN
"MDDATVLRKKGYIVGINLGKGSYAKVKSAYSERLKFNVAVKIIDRKKTPTDFVERFLPREMDILATVNHGSIIKTYEIFETSDGRIYIIMELGVQGDLLEFIKCQGALHEDVARKMFRQLSSAVKYCHDLDIVHRDLKCENLLLDKDFNIKLSDFGFSKRCLRDSNGRIILSKTFCGSAAYAAPEVLQSIPYQPKVYDIWSLGVILYIMVCGSMPYDDSDIRKMLRIQKEHRVDFPRSKNLTCECKDLIYRMLQPDVSQRLHIDEILSHSWLQPPKPKATSSASFKREGEGKYRAECKLDTKTGLRPDHRPDHKLGAKTQHRLLVVPENENRMEDRLAETSRAKDHHISGAEVGKAST",
# SCYL2_HUMAN
"MESMLNKLKSTVTKVTADVTSAVMGNPVTREFDVGRHIASGGNGLAWKIFNGTKKSTKQEVAVFVFDKKLIDKYQKFEKDQIIDSLKRGVQQLTRLRHPRLLTVQHPLEESRDCLAFCTEPVFASLANVLGNWENLPSPISPDIKDYKLYDVETKYGLLQVSEGLSFLHSSVKMVHGNITPENIILNKSGAWKIMGFDFCVSSTNPSEQEPKFPCKEWDPNLPSLCLPNPEYLAPEYILSVSCETASDMYSLGTVMYAVFNKGKPIFEVNKQDIYKSFSRQLDQLSRLGSSSLTNIPEEVREHVKLLLNVTPTVRPDADQMTKIPFFDDVGAVTLQYFDTLFQRDNLQKSQFFKGLPKVLPKLPKRVIVQRILPCLTSEFVNPDMVPFVLPNVLLIAEECTKEEYVKLILPELGPVFKQQEPIQILLIFLQKMDLLLTKTPPDEIKNSVLPMVYRALEAPSIQIQELCLNIIPTFANLIDYPSMKNALIPRIKNACLQTSSLAVRVNSLVCLGKILEYLDKWFVLDDILPFLQQIPSKEPAVLMGILGIYKCTFTHKKLGITKEQLAGKVLPHLIPLSIENNLNLNQFNSFISVIKEMLNRLESEHKTKLEQLHIMQEQQKSLDIGNQMNVSEEMKVTNIGNQQIDKVFNNIGADLLTGSESENKEDGLQNKHKRASLTLEEKQKLAKEQEQAQKLKSQQPLKPQVHTPVATVKQTKDLTDTLMDNMSSLTSLSVSTPKSSASSTFTSVPSMGIGMMFSTPTDNTKRNLTNGLNANMGFQTSGFNMPVNTNQNFYSSPSTVGVTKMTLGTPPTLPNFNALSVPPAGAKQTQQRPTDMSALNNLFGPQKPKVSMNQLSQQKPNQWLNQFVPPQGSPTMGSSVMGTQMNVIGQSAFGMQGNPFFNPQNFAQPPTTMTNSSSASNDLKDLFG",
# NEK8_HUMAN
"MEKYERIRVVGRGAFGIVHLCLRKADQKLVIIKQIPVEQMTKEERQAAQNECQVLKLLNHPNVIEYYENFLEDKALMIAMEYAPGGTLAEFIQKRCNSLLEEETILHFFVQILLALHHVHTHLILHRDLKTQNILLDKHRMVVKIGDFGISKILSSKSKAYTVVGTPCYISPELCEGKPYNQKSDIWALGCVLYELASLKRAFEAANLPALVLKIMSGTFAPISDRYSPELRQLVLSLLSLEPAQRPPLSHIMAQPLCIRALLNLHTDVGSVRMRRAEKSVAPSNTGSRTTSVRCRGIPRGPVRPAIPPPLSSVYAWGGGLGTPLRLPMLNTEVVQVAAGRTQKAGVTRSGRLILWEAPPLGAGGGSLLPGAVEQPQPQFISRFLEGQSGVTIKHVACGDFFTACLTDRGIIMTFGSGSNGCLGHGSLTDISQPTIVEALLGYEMVQVACGASHVLALSTERELFAWGRGDSGRLGLGTRESHSCPQQVPMPPGQEAQRVVCGIDSSMILTVPGQALACGSNRFNKLGLDHLSLGEEPVPHQQVEEALSFTLLGSAPLDQEPLLSIDLGTAHSAAVTASGDCYTFGSNQHGQLGTNTRRGSRAPCKVQGLEGIKMAMVACGDAFTVAIGAESEVYSWGKGARGRLGRRDEDAGLPRPVQLDETHPYTVTSVSCCHGNTLLAVRSVTDEPVPP",
# ARBK2_HUMAN
"MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKYLAERNEITFDKIFNQKIGFLLFKDFCLNEINEAVPQVKFYEEIKEYEKLDNEEDRLCRSRQIYDAYIMKELLSCSHPFSKQAVEHVQSHLSKKQVTSTLFQPYIEEICESLRGDIFQKFMESDKFTRFCQWKNVELNIHLTMNEFSVHRIIGRGGFGEVYGCRKADTGKMYAMKCLDKKRIKMKQGETLALNERIMLSLVSTGDCPFIVCMTYAFHTPDKLCFILDLMNGGDLHYHLSQHGVFSEKEMRFYATEIILGLEHMHNRFVVYRDLKPANILLDEHGHARISDLGLACDFSKKKPHASVGTHGYMAPEVLQKGTAYDSSADWFSLGCMLFKLLRGHSPFRQHKTKDKHEIDRMTLTVNVELPDTFSPELKSLLEGLLQRDVSKRLGCHGGGSQEVKEHSFFKGVDWQHVYLQKYPPPLIPPRGEVNAADAFDIGSFDEEDTKGIKLLDCDQELYKNFPLVISERWQQEVTETVYEAVNADTDKIEARKRAKNKQLGHEEDYALGKDCIMHGYMLKLGNPFLTQWQRRYFYLFPNRLEWRGEGESRQNLLTMEQILSVEETQIKDKKCILFRIKGGKQFVLQCESDPEFVQWKKELNETFKEAQRLLRRAPKFLNKPRSGTVELPKPSLCHRNSNGL",
# NRBP_HUMAN
"MSEGESQTVLSSGSDPKVESSSSAPGLTSVSPPVTSTTSAASPEEEEESEDESEILEESPCGRWQKRREEVNQRNVPGIDSAYLAMDTEEGVEVVWNEVQFSERKNYKLQEEKVRAVFDNLIQLEHLNIVKFHKYWADIKENKARVIFITEYMSSGSLKQFLKKTKKNHKTMNEKAWKRWCTQILSALSYLHSCDPPIIHGNLTCDTIFIQHNGLIKIGSVAPDTINNHVKTCREEQKNLHFFAPEYGEVTNVTTAVDIYSFGMCALEMAVLEIQGNGESSYVPQEAISSAIQLLEDPLQREFIQKCLQSEPARRPTARELLFHPALFEVPSLKLLAAHCIVGHQHMIPENALEEITKNMDTSAVLAEIPAGPGREPVQTLYSQSPALELDKFLEDVRNGIYPLTAFGLPRPQQPQQEEVTSPVVPPSVKTPTPEPAEVETRKVVLMQCNIESVEEGVKHHLTLLLKLEDKLNRHLSCDLMPNENIPELAAELVQLGFISEADQSRLTSLLEETLNKFNFARNSTLNSAAVTVSS",
# KPCD2_HUMAN
"MATAPSYPAGLPGSPGPGSPPPPGGLELQSPPPLLPQIPAPGSGVSFHIQIGLTREFVLLPAASELAHVKQLACSIVDQKFPECGFYGLYDKILLFKHDPTSANLLQLVRSSGDIQEGDLVEVVLSASATFEDFQIRPHALTVHSYRAPAFCDHCGEMLFGLVRQGLKCDGCGLNYHKRCAFSIPNNCSGARKRRLSSTSLASGHSVRLGTSESLPCTAEELSRSTTELLPRRPPSSSSSSSASSYTGRPIELDKMLLSKVKVPHTFLIHSYTRPTVCQACKKLLKGLFRQGLQCKDCKFNCHKRCATRVPNDCLGEALINGDVPMEEATDFSEADKSALMDESEDSGVIPGSHSENALHASEEEEGEGGKAQSSLGYIPLMRVVQSVRHTTRKSSTTLREGWVVHYSNKDTLRKRHYWRLDCKCITLFQNNTTNRYYKEIPLSEILTVESAQNFSLVPPGTNPHCFEIVTANATYFVGEMPGGTPGGPSGQGAEAARGWETAIRQALMPVILQDAPSAPGHAPHRQASLSISVSNSQIQENVDIATVYQIFPDEVLGSGQFGVVYGGKHRKTGRDVAVKVIDKLRFPTKQESQLRNEVAILQSLRHPGIVNLECMFETPEKVFVVMEKLHGDMLEMILSSEKGRLPERLTKFLITQILVALRHLHFKNIVHCDLKPENVLLASADPFPQVKLCDFGFARIIGEKSFRRSVVGTPAYLAPEVLLNQGYNRSLDMWSVGVIMYVSLSGTFPFNEDEDINDQIQNAAFMYPASPWSHISAGAIDLINNLLQVKMRKRYSVDKSLSHPWLQEYQTWLDLRELEGKMGERYITHESDDARWEQFAAEHPLPGSGLPTDRDLGGACPPQDHDMQGLAERISVL",
# ST32B_HUMAN
"MGGNHSHKPPVFDENEEVNFDHFQILRAIGKGSFGKVCIVQKRDTKKMYAMKYMNKQKCIERDEVRNVFRELQIMQGLEHPFLVNLWYSFQDEEDMFMVVDLLLGGDLRYHLQQNVHFTEGTVKLYICELALALEYLQRYHIIHRDIKPDNILLDEHGHVHITDFNIATVVKGAERASSMAGTKPYMAPEVFQVYMDRGPGYSYPVDWWSLGITAYELLRGWRPYEIHSVTPIDEILNMFKVERVHYSSTWCKGMVALLRKLLTKDPESRVSSLHDIQSVPYLADMNWDAVFKKALMPGFVPNKGRLNCDPTFELEEMILESKPLHKKKKRLAKNRSRDGTKDSCPLNGHLQHCLETVREEFIIFNREKLRRQQGQGSQLLDTDSRGGGQAQSKLQDGCNNNLLTHTCTRGCSS",
# KKCC2_HUMAN
"MSSCVSSQPSSNRAAPQDELGGRGSSSSESQKPCEALRGLSSLSIHLGMESFIVVTECEPGCAVDLGLARDRPLEADGQEVPLDTSGSQARPHLSGRKLSLQERSQGGLAAGGSLDMNGRCICPSLPYSPVSSPQSSPRLPRRPTVESHHVSITGMQDCVQLNQYTLKDEIGKGSYGVVKLAYNENDNTYYAMKVLSKKKLIRQAGFPRRPPPRGTRPAPGGCIQPRGPIEQVYQEIAILKKLDHPNVVKLVEVLDDPNEDHLYMVFELVNQGPVMEVPTLKPLSEDQARFYFQDLIKGIEYLHYQKIIHRDIKPSNLLVGEDGHIKIADFGVSNEFKGSDALLSNTVGTPAFMAPESLSETRKIFSGKALDVWAMGVTLYCFVFGQCPFMDERIMCLHSKIKSQALEFPDQPDIAEDLKDLITRMLDKNPESRIVVPEIKLHPWVTRHGAEPLPSEDENCTLVEVTEEEVENSVKHIPSLATVILVKTMIRKRSFGNPFEGSRREERSLSAPGNLLTKKPTRECESLSELKEARQRRQPPGHRPAPRGGGGSALVRGSPCVESCWAPAPGSPARMHPLRPEEAMEPE",
# CDK20_HUMAN
"MDQYCILGRIGEGAHGIVFKAKHVETGEIVALKKVALRRLEDGFPNQALREIKALQEMEDNQYVVQLKAVFPHGGGFVLAFEFMLSDLAEVVRHAQRPLAQAQVKSYLQMLLKGVAFCHANNIVHRDLKPANLLISASGQLKIADFGLARVFSPDGSRLYTHQVATRWYRAPELLYGARQYDQGVDLWSVGCIMGELLNGSPLFPGKNDIEQLCYVLRILGTPNPQVWPELTELPDYNKISFKEQVPMPLEEVLPDVSPQALDLLGQFLLYPPHQRIAASKALLHQYFFTAPLPAHPSELPIPQRLGGPAPKAHPGPPHIHDFHVDRPLEESLLNPELIRPFILEG",
# CLK4_HUMAN
"MRHSKRTHCPDWDSRESWGHESYRGSHKRKRRSHSSTQENRHCKPHHQFKESDCHYLEARSLNERDYRDRRYVDEYRNDYCEGYVPRHYHRDIESGYRIHCSKSSVRSRRSSPKRKRNRHCSSHQSRSKSHRRKRSRSIEDDEEGHLICQSGDVLRARYEIVDTLGEGAFGKVVECIDHGMDGMHVAVKIVKNVGRYREAARSEIQVLEHLNSTDPNSVFRCVQMLEWFDHHGHVCIVFELLGLSTYDFIKENSFLPFQIDHIRQMAYQICQSINFLHHNKLTHTDLKPENILFVKSDYVVKYNSKMKRDERTLKNTDIKVVDFGSATYDDEHHSTLVSTRHYRAPEVILALGWSQPCDVWSIGCILIEYYLGFTVFQTHDSKEHLAMMERILGPIPQHMIQKTRKRKYFHHNQLDWDEHSSAGRYVRRRCKPLKEFMLCHDEEHEKLFDLVRRMLEYDPTQRITLDEALQHPFFDLLKKK",
# CDK12_HUMAN
"MPNSERHGGKKDGSGGASGTLQPSSGGGSSNSRERHRLVSKHKRHKSKHSKDMGLVTPEAASLGTVIKPLVEYDDISSDSDTFSDDMAFKLDRRENDERRGSDRSDRLHKHRHHQHRRSRDLLKAKQTEKEKSQEVSSKSGSMKDRISGSSKRSNEETDDYGKAQVAKSSSKESRSSKLHKEKTRKERELKSGHKDRSKSHRKRETPKSYKTVDSPKRRSRSPHRKWSDSSKQDDSPSGASYGQDYDLSPSRSHTSSNYDSYKKSPGSTSRRQSVSPPYKEPSAYQSSTRSPSPYSRRQRSVSPYSRRRSSSYERSGSYSGRSPSPYGRRRSSSPFLSKRSLSRSPLPSRKSMKSRSRSPAYSRHSSSHSKKKRSSSRSRHSSISPVRLPLNSSLGAELSRKKKERAAAAAAAKMDGKESKGSPVFLPRKENSSVEAKDSGLESKKLPRSVKLEKSAPDTELVNVTHLNTEVKNSSDTGKVKLDENSEKHLVKDLKAQGTRDSKPIALKEEIVTPKETETSEKETPPPLPTIASPPPPLPTTTPPPQTPPLPPLPPIPALPQQPPLPPSQPAFSQVPASSTSTLPPSTHSKTSAVSSQANSQPPVQVSVKTQVSVTAAIPHLKTSTLPPLPLPPLLPGDDDMDSPKETLPSKPVKKEKEQRTRHLLTDLPLPPELPGGDLSPPDSPEPKAITPPQQPYKKRPKICCPRYGERRQTESDWGKRCVDKFDIIGIIGEGTYGQVYKAKDKDTGELVALKKVRLDNEKEGFPITAIREIKILRQLIHRSVVNMKEIVTDKQDALDFKKDKGAFYLVFEYMDHDLMGLLESGLVHFSEDHIKSFMKQLMEGLEYCHKKNFLHRDIKCSNILLNNSGQIKLADFGLARLYNSEESRPYTNKVITLWYRPPELLLGEERYTPAIDVWSCGCILGELFTKKPIFQANLELAQLELISRLCGSPCPAVWPDVIKLPYFNTMKPKKQYRRRLREEFSFIPSAALDLLDHMLTLDPSKRCTAEQTLQSDFLKDVELSKMAPPDLPHWQDCHELWSKKRRRQRQSGVVVEEPPPSKTSRKETTSGTSTEPVKNSSPAPPQPAPGKVESGAGDAIGLADITQQLNQSELAVLLNLLQSQTDLSIPQMAQLLNIHSNPEMQQQLEALNQSISALTEATSQQQDSETMAPEESLKEAPSAPVILPSAEQTTLEASSTPADMQNILAVLLSQLMKTQEPAGSLEENNSDKNSGPQGPRRTPTMPQEEAAACPPHILPPEKRPPEPPGPPPPPPPPPLVEGDLSSAPQELNPAVTAALLQLLSQPEAEPPGHLPHEHQALRPMEYSTRPRPNRTYGNTDGPETGFSAIDTDERNSGPALTESLVQTLVKNRTFSGSLSHLGESSSYQGTGSVQFPGDQDLRFARVPLALHPVVGQPFLKAEGSSNSVVHAETKLQNYGELGPGTTGASSSGAGLHWGGPTQSSAYGKLYRGPTRVPPRGGRGRGVPY",
# ST17A_HUMAN
"MIPLEKPGSGGSSPGATSGSGRAGRGLSGPCRPPPPPQARGLLTEIRAVVRTEPFQDGYSLCPGRELGRGKFAVVRKCIKKDSGKEFAAKFMRKRRKGQDCRMEIIHEIAVLELAQDNPWVINLHEVYETASEMILVLEYAAGGEIFDQCVADREEAFKEKDVQRLMRQILEGVHFLHTRDVVHLDLKPQNILLTSESPLGDIKIVDFGLSRILKNSEELREIMGTPEYVAPEILSYDPISMATDMWSIGVLTYVMLTGISPFLGNDKQETFLNISQMNLSYSEEEFDVLSESAVDFIRTLLVKKPEDRATAEECLKHPWLTQSSIQEPSFRMEKALEEANALQEGHSVPEINSDTDKSETKESIVTEELIVVTSYTLGQCRQSEKEKMEQKAISKRFKFEEPLLQEIPGEFIY",
# ST17B_HUMAN
"MSRRRFDCRSISGLLTTTPQIPIKMENFNNFYILTSKELGRGKFAVVRQCISKSTGQEYAAKFLKKRRRGQDCRAEILHEIAVLELAKSCPRVINLHEVYENTSEIILILEYAAGGEIFSLCLPELAEMVSENDVIRLIKQILEGVYYLHQNNIVHLDLKPQNILLSSIYPLGDIKIVDFGMSRKIGHACELREIMGTPEYLAPEILNYDPITTATDMWNIGIIAYMLLTHTSPFVGEDNQETYLNISQVNVDYSEETFSSVSQLATDFIQSLLVKNPEKRPTAEICLSHSWLQQWDFENLFHPEETSSSSQTQDHSVRSSEDKTSKSSCNGTCGDREDKENIPEDSSMVSKRFRFDDSLPNPHELVSDLLC",
# DYRK3_HUMAN
"MGGTARGPGRKDAGPPGAGLPPQQRRLGDGVYDTFMMIDETKCPPCSNVLCNPSEPPPPRRLNMTTEQFTGDHTQHFLDGGEMKVEQLFQEFGNRKSNTIQSDGISDSEKCSPTVSQGKSSDCLNTVKSNSSSKAPKVVPLTPEQALKQYKHHLTAYEKLEIINYPEIYFVGPNAKKRHGVIGGPNNGGYDDADGAYIHVPRDHLAYRYEVLKIIGKGSFGQVARVYDHKLRQYVALKMVRNEKRFHRQAAEEIRILEHLKKQDKTGSMNVIHMLESFTFRNHVCMAFELLSIDLYELIKKNKFQGFSVQLVRKFAQSILQSLDALHKNKIIHCDLKPENILLKHHGRSSTKVIDFGSSCFEYQKLYTYIQSRFYRAPEIILGSRYSTPIDIWSFGCILAELLTGQPLFPGEDEGDQLACMMELLGMPPPKLLEQSKRAKYFINSKGIPRYCSVTTQADGRVVLVGGRSRRGKKRGPPGSKDWGTALKGCDDYLFIEFLKRCLHWDPSARLTPAQALRHPWISKSVPRPLTTIDKVSGKRVVNPASAFQGLGSKLPPVVGIANKLKANLMSETNGSIPLCSVLPKLIS",
# KPCD3_HUMAN
"MSANNSPPSAQKSVLPTAIPAVLPAASPCSSPKTGLSARLSNGSFSAPSLTNSRGSVHTVSFLLQIGLTRESVTIEAQELSLSAVKDLVCSIVYQKFPECGFFGMYDKILLFRHDMNSENILQLITSADEIHEGDLVEVVLSALATVEDFQIRPHTLYVHSYKAPTFCDYCGEMLWGLVRQGLKCEGCGLNYHKRCAFKIPNNCSGVRKRRLSNVSLPGPGLSVPRPLQPEYVALPSEESHVHQEPSKRIPSWSGRPIWMEKMVMCRVKVPHTFAVHSYTRPTICQYCKRLLKGLFRQGMQCKDCKFNCHKRCASKVPRDCLGEVTFNGEPSSLGTDTDIPMDIDNNDINSDSSRGLDDTEEPSPPEDKMFFLDPSDLDVERDEEAVKTISPSTSNNIPLMRVVQSIKHTKRKSSTMVKEGWMVHYTSRDNLRKRHYWRLDSKCLTLFQNESGSKYYKEIPLSEILRISSPRDFTNISQGSNPHCFEIITDTMVYFVGENNGDSSHNPVLAATGVGLDVAQSWEKAIRQALMPVTPQASVCTSPGQGKDHKDLSTSISVSNCQIQENVDISTVYQIFADEVLGSGQFGIVYGGKHRKTGRDVAIKVIDKMRFPTKQESQLRNEVAILQNLHHPGIVNLECMFETPERVFVVMEKLHGDMLEMILSSEKSRLPERITKFMVTQILVALRNLHFKNIVHCDLKPENVLLASAEPFPQVKLCDFGFARIIGEKSFRRSVVGTPAYLAPEVLRSKGYNRSLDMWSVGVIIYVSLSGTFPFNEDEDINDQIQNAAFMYPPNPWREISGEAIDLINNLLQVKMRKRYSVDKSLSHPWLQDYQTWLDLREFETRIGERYITHESDDARWEIHAYTHNLVYPKHFIMAPNPDDMEEDP",
# E2AK4_HUMAN
"MAGGRGAPGRGRDEPPESYPQRQDHELQALEAIYGADFQDLRPDACGPVKEPPEINLVLYPQGLTGEEVYVKVDLRVKCPPTYPDVVPEIELKNAKGLSNESVNLLKSRLEELAKKHCGEVMIFELAYHVQSFLSEHNKPPPKSFHEEMLERRAQEEQQRLLEAKRKEEQEQREILHEIQRRKEEIKEEKKRKEMAKQERLEIASLSNQDHTSKKDPGGHRTAAILHGGSPDFVGNGKHRANSSGRSRRERQYSVCNSEDSPGSCEILYFNMGSPDQLMVHKGKCIGSDEQLGKLVYNALETATGGFVLLYEWVLQWQKKMGPFLTSQEKEKIDKCKKQIQGTETEFNSLVKLSHPNVVRYLAMNLKEQDDSIVVDILVEHISGVSLAAHLSHSGPIPVHQLRRYTAQLLSGLDYLHSNSVVHKVLSASNVLVDAEGTVKITDYSISKRLADICKEDVFEQTRVRFSDNALPYKTGKKGDVWRLGLLLLSLSQGQECGEYPVTIPSDLPADFQDFLKKCVCLDDKERWSPQQLLKHSFINPQPKMPLVEQSPEDSEGQDYVETVIPSNRLPSAAFFSETQRQFSRYFIEFEELQLLGKGAFGAVIKVQNKLDGCCYAVKRIPINPASRQFRRIKGEVTLLSRLHHENIVRYYNAWIERHERPAGPGTPPPDSGPLAKDDRAARGQPASDTDGLDSVEAAAPPPILSSSVEWSTSGERSASARFPATGPGSSDDEDDDEDEHGGVFSQSFLPASDSESDIIFDNEDENSKSQNQDEDCNEKNGCHESEPSVTTEAVHYLYIQMEYCEKSTLRDTIDQGLYRDTVRLWRLFREILDGLAYIHEKGMIHRDLKPVNIFLDSDDHVKIGDFGLATDHLAFSADSKQDDQTGDLIKSDPSGHLTGMVGTALYVSPEVQGSTKSAYNQKVDLFSLGIIFFEMSYHPMVTASERIFVLNQLRDPTSPKFPEDFDDGEHAKQKSVISWLLNHDPAKRPTATELLKSELLPPPQMEESELHEVLHHTLTNVDGKAYRTMMAQIFSQRISPAIDYTYDSDILKGNFSIRTAKMQQHVCETIIRIFKRHGAVQLCTPLLLPRNRQIYEHNEAALFMDHSGMLVMLPFDLRIPFARYVARNNILNLKRYCIERVFRPRKLDRFHPKELLECAFDIVTSTTNSFLPTAEIIYTIYEIIQEFPALQERNYSIYLNHTMLLKAILLHCGIPEDKLSQVYIILYDAVTEKLTRREVEAKFCNLSLSSNSLCRLYKFIEQKGDLQDLMPTINSLIKQKTGIAQLVKYGLKDLEEVVGLLKKLGIKLQVLINLGLVYKVQQHNGIIFQFVAFIKRRQRAVPEILAAGGRYDLLIPQFRGPQALGPVPTAIGVSIAIDKISAAVLNMEESVTISSCDLLVVSVGQMSMSRAINLTQKLWTAGITAEIMYDWSQSQEELQEYCRHHEITYVALVSDKEGSHVKVKSFEKERQTEKRVLETELVDHVLQKLRTKVTDERNGREASDNLAVQNLKGSFSNASGLFEIHGATVVPIVSVLAPEKLSASTRRRYETQVQTRLQTSLANLHQKSSEIEILAVDLPKETILQFLSLEWDADEQAFNTTVKQLLSRLPKQRYLKLVCDEIYNIKVEKKVSVLFLYSYRDDYYRILF",
# KS6R_HUMAN
"MGAVSCRQGQHTQQGEHTRVAVPHKQGGNIRGPWARGWKSLWTGLGTIRSDLEELWELRGHHYLHQESLKPAPVLVEKPLPEWPVPQFINLFLPEFPIRPIRGQQQLKILGLVAKGSFGTVLKVLDCTQKAVFAVKVVPKVKVLQRDTVRQCKEEVSIQRQINHPFVHSLGDSWQGKRHLFIMCSYCSTDLYSLWSAVGCFPEASIRLFAAELVLVLCYLHDLGIMHRDVKMENILLDERGHLKLTDFGLSRHVPQGAQAYTICGTLQYMAPEVLSGGPYNHAADWWSLGVLLFSLATGKFPVAAERDHVAMLASVTHSDSEIPASLNQGLSLLLHELLCQNPLHRLRYLHHFQVHPFFRGVAFDPELLQKQPVNFVTETQATQPSSAETMPFDDFDCDLESFLLYPIPA",
# STK40_HUMAN
"MKRRASDRGAGETSARAKALGSGISGNNAKRAGPFILGPRLGNSPVPSIVQCLARKDGTDDFYQLKILTLEERGDQGIESQEERQGKMLLHTEYSLLSLLHTQDGVVHHHGLFQDRTCEIVEDTESSRMVKKMKKRICLVLDCLCAHDFSDKTADLINLQHYVIKEKRLSERETVVIFYDVVRVVEALHQKNIVHRDLKLGNMVLNKRTHRITITNFCLGKHLVSEGDLLKDQRGSPAYISPDVLSGRPYRGKPSDMWALGVVLFTMLYGQFPFYDSIPQELFRKIKAAEYTIPEDGRVSENTVCLIRKLLVLDPQQRLAAADVLEALSAIIASWQSLSSLSGPLQVVPDIDDQMSNADSSQEAKVTEECSQYEFENYMRQQLLLAEEKSSIHDARSWVPKRQFGSAPPVRRLGHDAQPMTSLDTAILAQRYLRK",
# STK35_HUMAN
"MGHQESPLARAPAGGAAYVKRLCKGLSWREHVESHGSLGAQASPASAAAAEGSATRRARAATSRAARSRRQPGPGADHPQAGAPGGKRAARKWRCAGQVTIQGPAPPRPRAGRRDEAGGARAAPLLLPPPPAAMETGKDGARRGTQSPERKRRSPVPRAPSTKLRPAAAARAMDPVAAEAPGEAFLARRRPEGGGGSARPRYSLLAEIGRGSYGVVYEAVAGRSGARVAVKKIRCDAPENVELALAEFWALTSLKRRHQNVVQFEECVLQRNGLAQRMSHGNKSSQLYLRLVETSLKGERILGYAEEPCYLWFVMEFCEGGDLNQYVLSRRPDPATNKSFMLQLTSAIAFLHKNHIVHRDLKPDNILITERSGTPILKVADFGLSKVCAGLAPRGKEGNQDNKNVNVNKYWLSSACGSDFYMAPEVWEGHYTAKADIFALGIIIWAMIERITFIDSETKKELLGTYIKQGTEIVPVGEALLENPKMELHIPQKRRTSMSEGIKQLLKDMLAANPQDRPDAFELETRMDQVTCAA",
# TNI3K_HUMAN
"MGNYKSRPTQTCTDEWKKKVSESYVITIERLEDDLQIKEKELTELRNIFGSDEAFSKVNLNYRTENGLSLLHLCCICGGKKSHIRTLMLKGLRPSRLTRNGFTALHLAVYKDNAELITSLLHSGADIQQVGYGGLTALHIATIAGHLEAADVLLQHGANVNIQDAVFFTPLHIAAYYGHEQVTRLLLKFGADVNVSGEVGDRPLHLASAKGFLNIAKLLMEEGSKADVNAQDNEDHVPLHFCSRFGHHDIVKYLLQSDLEVQPHVVNIYGDTPLHLACYNGKFEVAKEIIQISGTESLTKENIFSETAFHSACTYGKSIDLVKFLLDQNVININHQGRDGHTGLHSACYHGHIRLVQFLLDNGADMNLVACDPSRSSGEKDEQTCLMWAYEKGHDAIVTLLKHYKRPQDELPCNEYSQPGGDGSYVSVPSPLGKIKSMTKEKADILLLRAGLPSHFHLQLSEIEFHEIIGSGSFGKVYKGRCRNKIVAIKRYRANTYCSKSDVDMFCREVSILCQLNHPCVIQFVGACLNDPSQFAIVTQYISGGSLFSLLHEQKRILDLQSKLIIAVDVAKGMEYLHNLTQPIIHRDLNSHNILLYEDGHAVVADFGESRFLQSLDEDNMTKQPGNLRWMAPEVFTQCTRYTIKADVFSYALCLWEILTGEIPFAHLKPAAAAADMAYHHIRPPIGYSIPKPISSLLIRGWNACPEGRPEFSEVVMKLEECLCNIELMSPASSNSSGSLSPSSSSDCLVNRGGPGRSHVAALRSRFELEYALNARSYAALSQSAGQYSSQGLSLEEMKRSLQYTPIDKYGYVSDPMSSMHFHSCRNSSSFEDSS",
# HIPK2_HUMAN
"MAPVYEGMASHVQVFSPHTLQSSAFCSVKKLKIEPSSNWDMTGYGSHSKVYSQSKNIPLSQPATTTVSTSLPVPNPSLPYEQTIVFPGSTGHIVVTSASSTSVTGQVLGGPHNLMRRSTVSLLDTYQKCGLKRKSEEIENTSSVQIIEEHPPMIQNNASGATVATATTSTATSKNSGSNSEGDYQLVQHEVLCSMTNTYEVLEFLGRGTFGQVVKCWKRGTNEIVAIKILKNHPSYARQGQIEVSILARLSTESADDYNFVRAYECFQHKNHTCLVFEMLEQNLYDFLKQNKFSPLPLKYIRPVLQQVATALMKLKSLGLIHADLKPENIMLVDPSRQPYRVKVIDFGSASHVSKAVCSTYLQSRYYRAPEIILGLPFCEAIDMWSLGCVIAELFLGWPLYPGASEYDQIRYISQTQGLPAEYLLSAGTKTTRFFNRDTDSPYPLWRLKTPDDHEAETGIKSKEARKYIFNCLDDMAQVNMTTDLEGSDMLVEKADRREFIDLLKKMLTIDADKRITPIETLNHPFVTMTHLLDFPHSTHVKSCFQNMEICKRRVNMYDTVNQSKTPFITHVAPSTSTNLTMTFNNQLTTVHNQAPSSTSATISLANPEVSILNYPSTLYQPSAASMAAVAQRSMPLQTGTAQICARPDPFQQALIVCPPGFQGLQASPSKHAGYSVRMENAVPIVTQAPGAQPLQIQPGLLAQQAWPSGTQQILLPPAWQQLTGVATHTSVQHATVIPETMAGTQQLADWRNTHAHGSHYNPIMQQPALLTGHVTLPAAQPLNVGVAHVMRQQPTSTTSSRKSKQHQSSVRNVSTCEVSSSQAISSPQRSKRVKENTPPRCAMVHSSPACSTSVTCGWGDVASSTTRERQRQTIVIPDTPSPTVSVITISSDTDEEEEQKHAPTSTVSKQRKNVISCVTVHDSPYSDSSSNTSPYSVQQRAGHNNANAFDTKGSLENHCTGNPRTIIVPPLKTQASEVLVECDSLVPVNTSHHSSSYKSKSSSNVTSTSGHSSGSSSGAITYRQQRPGPHFQQQQPLNLSQAQQHITTDRTGSHRRQQAYITPTMAQAPYSFPHNSPSHGTVHPHLAAAAAAAHLPTQPHLYTYTAPAALGSTGTVAHLVASQGSARHTVQHTAYPASIVHQVPVSMGPRVLPSPTIHPSQYPAQFAHQTYISASPASTVYTGYPLSPAKVNQYPYI",
# E2AK1_HUMAN
"MQGGNSGVRKREEEGDGAGAVAAPPAIDFPAEGPDPEYDESDVPAEIQVLKEPLQQPTFPFAVANQLLLVSLLEHLSHVHEPNPLRSRQVFKLLCQTFIKMGLLSSFTCSDEFSSLRLHHNRAITHLMRSAKERVRQDPCEDISRIQKIRSREVALEAQTSRYLNEFEELAILGKGGYGRVYKVRNKLDGQYYAIKKILIKGATKTVCMKVLREVKVLAGLQHPNIVGYHTAWIEHVHVIQPRADRAAIELPSLEVLSDQEEDREQCGVKNDESSSSSIIFAEPTPEKEKRFGESDTENQNNKSVKYTTNLVIRESGELESTLELQENGLAGLSASSIVEQQLPLRRNSHLEESFTSTEESSEENVNFLGQTEAQYHLMLHIQMQLCELSLWDWIVERNKRGREYVDESACPYVMANVATKIFQELVEGVFYIHNMGIVHRDLKPRNIFLHGPDQQVKIGDFGLACTDILQKNTDWTNRNGKRTPTHTSRVGTCLYASPEQLEGSEYDAKSDMYSLGVVLLELFQPFGTEMERAEVLTGLRTGQLPESLRKRCPVQAKYIQHLTRRNSSQRPSAIQLLQSELFQNSGNVNLTLQMKIIEQEKEIAELKKQLNLLSQDKGVRDDGKDGGVG",
# CILK1_HUMAN
"MNRYTTIRQLGDGTYGSVLLGRSIESGELIAIKKMKRKFYSWEECMNLREVKSLKKLNHANVVKLKEVIRENDHLYFIFEYMKENLYQLIKERNKLFPESAIRNIMYQILQGLAFIHKHGFFHRDLKPENLLCMGPELVKIADFGLAREIRSKPPYTDYVSTRWYRAPEVLLRSTNYSSPIDVWAVGCIMAEVYTLRPLFPGASEIDTIFKICQVLGTPKKTDWPEGYQLSSAMNFRWPQCVPNNLKTLIPNASSEAVQLLRDMLQWDPKKRPTASQALRYPYFQVGHPLGSTTQNLQDSEKPQKGILEKAGPPPYIKPVPPAQPPAKPHTRISSRQHQASQPPLHLTYPYKAEVSRTDHPSHLQEDKPSPLLFPSLHNKHPQSKITAGLEHKNGEIKPKSRRRWGLISRSTKDSDDWADLDDLDFSPSLSRIDLKNKKRQSDDTLCRFESVLDLKPSEPVGTGNSAPTQTSYQRRDTPTLRSAAKQHYLKHSRYLPGISIRNGILSNPGKEFIPPNPWSSSGLSGKSSGTMSVISKVNSVGSSSTSSSGLTGNYVPSFLKKEIGSAMQRVHLAPIPDPSPGYSSLKAMRPHPGRPFFHTQPRSTPGLIPRPPAAQPVHGRTDWASKYASRR",
# ERN2_HUMAN
"MASAVRGSRPWPRLGLQLQFAALLLGTLSPQVHTLRPENLLLVSTLDGSLHALSKQTGDLKWTLRDDPVIEGPMYVTEMAFLSDPADGSLYILGTQKQQGLMKLPFTIPELVHASPCRSSDGVFYTGRKQDAWFVVDPESGETQMTLTTEGPSTPRLYIGRTQYTVTMHDPRAPALRWNTTYRRYSAPPMDGSPGKYMSHLASCGMGLLLTVDPGSGTVLWTQDLGVPVMGVYTWHQDGLRQLPHLTLARDTLHFLALRWGHIRLPASGPRDTATLFSTLDTQLLMTLYVGKDETGFYVSKALVHTGVALVPRGLTLAPADGPTTDEVTLQVSGEREGSPSTAVRYPSGSVALPSQWLLIGHHELPPVLHTTMLRVHPTLGSGTAETRPPENTQAPAFFLELLSLSREKLWDSELHPEEKTPDSYLGLGPQDLLAASLTAVLLGGWILFVMRQQQPQVVEKQQETPLAPADFAHISQDAQSLHSGASRRSQKRLQSPSKQAQPLDDPEAEQLTVVGKISFNPKDVLGRGAGGTFVFRGQFEGRAVAVKRLLRECFGLVRREVQLLQESDRHPNVLRYFCTERGPQFHYIALELCRASLQEYVENPDLDRGGLEPEVVLQQLMSGLAHLHSLHIVHRDLKPGNILITGPDSQGLGRVVLSDFGLCKKLPAGRCSFSLHSGIPGTEGWMAPELLQLLPPDSPTSAVDIFSAGCVFYYVLSGGSHPFGDSLYRQANILTGAPCLAHLEEEVHDKVVARDLVGAMLSPLPQPRPSAPQVLAHPFFWSRAKQLQFFQDVSDWLEKESEQEPLVRALEAGGCAVVRDNWHEHISMPLQTDLRKFRSYKGTSVRDLLRAVRNKKHHYRELPVEVRQALGQVPDGFVQYFTNRFPRLLLHTHRAMRSCASESLFLPYYPPDSEARRPCPGATGR",
# PASK_HUMAN
"MEDGGLTAFEEDQRCLSQSLPLPVSAEGPAAQTTAEPSRSFSSAHRHLSRRNGLSRLCQSRTALSEDRWSSYCLSSLAAQNICTSKLHCPAAPEHTDPSEPRGSVSCCSLLRGLSSGWSSPLLPAPVCNPNKAIFTVDAKTTEILVANDKACGLLGYSSQDLIGQKLTQFFLRSDSDVVEALSEEHMEADGHAAVVFGTVVDIISRSGEKIPVSVWMKRMRQERRLCCVVVLEPVERVSTWVAFQSDGTVTSCDSLFAHLHGYVSGEDVAGQHITDLIPSVQLPPSGQHIPKNLKIQRSVGRARDGTTFPLSLKLKSQPSSEEATTGEAAPVSGYRASVWVFCTISGLITLLPDGTIHGINHSFALTLFGYGKTELLGKNITFLIPGFYSYMDLAYNSSLQLPDLASCLDVGNESGCGERTLDPWQGQDPAEGGQDPRINVVLAGGHVVPRDEIRKLMESQDIFTGTQTELIAGGQLLSCLSPQPAPGVDNVPEGSLPVHGEQALPKDQQITALGREEPVAIESPGQDLLGESRSEPVDVKPFASCEDSEAPVPAEDGGSDAGMCGLCQKAQLERMGVSGPSGSDLWAGAAVAKPQAKGQLAGGSLLMHCPCYGSEWGLWWRSQDLAPSPSGMAGLSFGTPTLDEPWLGVENDREELQTCLIKEQLSQLSLAGALDVPHAELVPTECQAVTAPVSSCDLGGRDLCGGCTGSSSACYALATDLPGGLEAVEAQEVDVNSFSWNLKELFFSDQTDQTSSNCSCATSELRETPSSLAVGSDPDVGSLQEQGSCVLDDRELLLLTGTCVDLGQGRRFRESCVGHDPTEPLEVCLVSSEHYAASDRESPGHVPSTLDAGPEDTCPSAEEPRLNVQVTSTPVIVMRGAAGLQREIQEGAYSGSCYHRDGLRLSIQFEVRRVELQGPTPLFCCWLVKDLLHSQRDSAARTRLFLASLPGSTHSTAAELTGPSLVEVLRARPWFEEPPKAVELEGLAACEGEYSQKYSTMSPLGSGAFGFVWTAVDKEKNKEVVVKFIKKEKVLEDCWIEDPKLGKVTLEIAILSRVEHANIIKVLDIFENQGFFQLVMEKHGSGLDLFAFIDRHPRLDEPLASYIFRQLVSAVGYLRLKDIIHRDIKDENIVIAEDFTIKLIDFGSAAYLERGKLFYTFCGTIEYCAPEVLMGNPYRGPELEMWSLGVTLYTLVFEENPFCELEETVEAAIHPPYLVSKELMSLVSGLLQPVPERRTTLEKLVTDPWVTQPVNLADYTWEEVFRVNKPESGVLSAASLEMGNRSLSDVAQAQELCGGPVPGEAPNGQGCLHPGDPRLLTS",
# ST38L_HUMAN
"MAMTAGTTTTFPMSNHTRERVTVAKLTLENFYSNLILQHEERETRQKKLEVAMEEEGLADEEKKLRRSQHARKETEFLRLKRTRLGLDDFESLKVIGRGAFGEVRLVQKKDTGHIYAMKILRKSDMLEKEQVAHIRAERDILVEADGAWVVKMFYSFQDKRNLYLIMEFLPGGDMMTLLMKKDTLTEEETQFYISETVLAIDAIHQLGFIHRDIKPDNLLLDAKGHVKLSDFGLCTGLKKAHRTEFYRNLTHNPPSDFSFQNMNSKRKAETWKKNRRQLAYSTVGTPDYIAPEVFMQTGYNKLCDWWSLGVIMYEMLIGYPPFCSETPQETYRKVMNWKETLVFPPEVPISEKAKDLILRFCIDSENRIGNSGVEEIKGHPFFEGVDWEHIRERPAAIPIEIKSIDDTSNFDDFPESDILQPVPNTTEPDYKSKDWVFLNYTYKRFEGLTQRGSIPTYMKAGKL",
# SIK3_HUMAN
"MAAAAASGAGGAAGAGTGGAGPAGRLLPPPAPGSPAAPAAVSPAAGQPRPPAPASRGPMPARIGYYEIDRTIGKGNFAVVKRATHLVTKAKVAIKIIDKTQLDEENLKKIFREVQIMKMLCHPHIIRLYQVMETERMIYLVTEYASGGEIFDHLVAHGRMAEKEARRKFKQIVTAVYFCHCRNIVHRDLKAENLLLDANLNIKIADFGFSNLFTPGQLLKTWCGSPPYAAPELFEGKEYDGPKVDIWSLGVVLYVLVCGALPFDGSTLQNLRARVLSGKFRIPFFMSTECEHLIRHMLVLDPNKRLSMEQICKHKWMKLGDADPNFDRLIAECQQLKEERQVDPLNEDVLLAMEDMGLDKEQTLQSLRSDAYDHYSAIYSLLCDRHKRHKTLRLGALPSMPRALAFQAPVNIQAEQAGTAMNISVPQVQLINPENQIVEPDGTLNLDSDEGEEPSPEALVRYLSMRRHTVGVADPRTEVMEDLQKLLPGFPGVNPQAPFLQVAPNVNFMHNLLPMQNLQPTGQLEYKEQSLLQPPTLQLLNGMGPLGRRASDGGANIQLHAQQLLKRPRGPSPLVTMTPAVPAVTPVDEESSDGEPDQEAVQSSTYKDSNTLHLPTERFSPVRRFSDGAASIQAFKAHLEKMGNNSSIKQLQQECEQLQKMYGGQIDERTLEKTQQQHMLYQQEQHHQILQQQIQDSICPPQPSPPLQAACENQPALLTHQLQRLRIQPSSPPPNHPNNHLFRQPSNSPPPMSSAMIQPHGAASSSQFQGLPSRSAIFQQQPENCSSPPNVALTCLGMQQPAQSQQVTIQVQEPVDMLSNMPGTAAGSSGRGISISPSAGQMQMQHRTNLMATLSYGHRPLSKQLSADSAEAHSLNVNRFSPANYDQAHLHPHLFSDQSRGSPSSYSPSTGVGFSPTQALKVPPLDQFPTFPPSAHQQPPHYTTSALQQALLSPTPPDYTRHQQVPHILQGLLSPRHSLTGHSDIRLPPTEFAQLIKRQQQQRQQQQQQQQQQEYQELFRHMNQGDAGSLAPSLGGQSMTERQALSYQNADSYHHHTSPQHLLQIRAQECVSQASSPTPPHGYAHQPALMHSESMEEDCSCEGAKDGFQDSKSSSTLTKGCHDSPLLLSTGGPGDPESLLGTVSHAQELGIHPYGHQPTAAFSKNKVPSREPVIGNCMDRSSPGQAVELPDHNGLGYPARPSVHEHHRPRALQRHHTIQNSDDAYVQLDNLPGMSLVAGKALSSARMSDAVLSQSSLMGSQQFQDGENEECGASLGGHEHPDLSDGSQHLNSSCYPSTCITDILLSYKHPEVSFSMEQAGV",
# HUNK_HUMAN
"MPAAAGDGLLGEPAAPGGGGGAEDAARPAAACEGSFLPAWVSGVPRERLRDFQHHKRVGNYLIGSRKLGEGSFAKVREGLHVLTGEKVAIKVIDKKRAKKDTYVTKNLRREGQIQQMIRHPNITQLLDILETENSYYLVMELCPGGNLMHKIYEKKRLEESEARRYIRQLISAVEHLHRAGVVHRDLKIENLLLDEDNNIKLIDFGLSNCAGILGYSDPFSTQCGSPAYAAPELLARKKYGPKIDVWSIGVNMYAMLTGTLPFTVEPFSLRALYQKMVDKEMNPLPTQLSTGAISFLRSLLEPDPVKRPNIQQALANRWLNENYTGKVPCNVTYPNRISLEDLSPSVVLHMTEKLGYKNSDVINTVLSNRACHILAIYFLLNKKLERYLSGKSDIQDSLCYKTRLYQIEKYRAPKESYEASLDTWTRDLEFHAVQDKKPKEQEKRGDFLHRPFSKKLDKNLPSHKQPSGSLMTQIQNTKALLKDRKASKSSFPDKDSFGCRNIFRKTSDSNCVASSSMEFIPVPPPRTPRIVKKPEPHQPGPGSTGIPHKEDPLMLDMVRSFESVDRDDHVEVLSPSHHYRILNSPVSLARRNSSERTLSPGLPSGSMSPLHTPLHPTLVSFAHEDKNSPPKEEGLCCPPPVPSNGPMQPLGSPNCVKSRGRFPMMGIGQMLRKRHQSLQPSADRPLEASLPPLQPLAPVNLAFDMADGVKTQC",
# M3K6_HUMAN
"MAGPCPRSGAERAGSCWQDPLAVALSRGRQLAAPPGRGCARSRPLSVVYVLTREPQPGLEPREGTEAEPLPLRCLREACAQVPRPRPPPQLRSLPFGTLELGDTAALDAFYNADVVVLEVSSSLVQPSLFYHLGVRESFSMTNNVLLCSQADLPDLQALREDVFQKNSDCVGSYTLIPYVVTATGRVLCGDAGLLRGLADGLVQAGVGTEALLTPLVGRLARLLEATPTDSCGYFRETIRRDIRQARERFSGPQLRQELARLQRRLDSVELLSPDIIMNLLLSYRDVQDYSAIIELVETLQALPTCDVAEQHNVCFHYTFALNRRNRPGDRAKALSVLLPLVQLEGSVAPDLYCMCGRIYKDMFFSSGFQDAGHREQAYHWYRKAFDVEPSLHSGINAAVLLIAAGQHFEDSKELRLIGMKLGCLLARKGCVEKMQYYWDVGFYLGAQILANDPTQVVLAAEQLYKLNAPIWYLVSVMETFLLYQHFRPTPEPPGGPPRRAHFWLHFLLQSCQPFKTACAQGDQCLVLVLEMNKVLLPAKLEVRGTDPVSTVTLSLLEPETQDIPSSWTFPVASICGVSASKRDERCCFLYALPPAQDVQLCFPSVGHCQWFCGLIQAWVTNPDSTAPAEEAEGAGEMLEFDYEYTETGERLVLGKGTYGVVYAGRDRHTRVRIAIKEIPERDSRFSQPLHEEIALHRRLRHKNIVRYLGSASQGGYLKIFMEEVPGGSLSSLLRSVWGPLKDNESTISFYTRQILQGLGYLHDNHIVHRDIKGDNVLINTFSGLLKISDFGTSKRLAGITPCTETFTGTLQYMAPEIIDQGPRGYGKAADIWSLGCTVIEMATGRPPFHELGSPQAAMFQVGMYKVHPPMPSSLSAEAQAFLLRTFEPDPRLRASAQTLLGDPFLQPGKRSRSPSSPRHAPRPSDAPSASPTPSANSTTQSQTFPCPQAPSQHPPSPPKRCLSYGGTSQLRVPEEPAAEEPASPEESSGLSLLHQESKRRAMLAAVLEQELPALAENLHQEQKQEQGARLGRNHVEELLRCLGAHIHTPNRRQLAQELRALQGRLRAQGLGPALLHRPLFAFPDAVKQILRKRQIRPHWMFVLDSLLSRAVRAALGVLGPEVEKEAVSPRSEELSNEGDSQQSPGQQSPLPVEPEQGPAPLMVQLSLLRAETDRLREILAGKEREYQALVQRALQRLNEEARTYVLAPEPPTALSTDQGLVQWLQELNVDSGTIQMLLNHSFTLHTLLTYATRDDLIYTRIRGGMVCRIWRAILAQRAGSTPVTSGP",
# M3K20_HUMAN
"MSSLGASFVQIKFDDLQFFENCGGGSFGSVYRAKWISQDKEVAVKKLLKIEKEAEILSVLSHRNIIQFYGVILEPPNYGIVTEYASLGSLYDYINSNRSEEMDMDHIMTWATDVAKGMHYLHMEAPVKVIHRDLKSRNVVIAADGVLKICDFGASRFHNHTTHMSLVGTFPWMAPEVIQSLPVSETCDTYSYGVVLWEMLTREVPFKGLEGLQVAWLVVEKNERLTIPSSCPRSFAELLHQCWEADAKKRPSFKQIISILESMSNDTSLPDKCNSFLHNKAEWRCEIEATLERLKKLERDLSFKEQELKERERRLKMWEQKLTEQSNTPLLPSFEIGAWTEDDVYCWVQQLVRKGDSSAEMSVYASLFKENNITGKRLLLLEEEDLKDMGIVSKGHIIHFKSAIEKLTHDYINLFHFPPLIKDSGGEPEENEEKIVNLELVFGFHLKPGTGPQDCKWKMYMEMDGDEIAITYIKDVTFNTNLPDAEILKMTKPPFVMEKWIVGIAKSQTVECTVTYESDVRTPKSTKHVHSIQWSRTKPQDEVKAVQLAIQTLFTNSDGNPGSRSDSSADCQWLDTLRMRQIASNTSLQRSQSNPILGSPFFSHFDGQDSYAAAVRRPQVPIKYQQITPVNQSRSSSPTQYGLTKNFSSLHLNSRDSGFSSGNTDTSSERGRYSDRSRNKYGRGSISLNSSPRGRYSGKSQHSTPSRGRYPGKFYRVSQSALNPHQSPDFKRSPRDLHQPNTIPGMPLHPETDSRASEEDSKVSEGGWTKVEYRKKPHRPSPAKTNKERARGDHRGWRNF",
# MOK_HUMAN
"MKNYKAIGKIGEGTFSEVMKMQSLRDGNYYACKQMKQRFESIEQVNNLREIQALRRLNPHPNILMLHEVVFDRKSGSLALICELMDMNIYELIRGRRYPLSEKKIMHYMYQLCKSLDHIHRNGIFHRDVKPENILIKQDVLKLGDFGSCRSVYSKQPYTEYISTRWYRAPECLLTDGFYTYKMDLWSAGCVFYEIASLQPLFPGVNELDQISKIHDVIGTPAQKILTKFKQSRAMNFDFPFKKGSGIPLLTTNLSPQCLSLLHAMVAYDPDERIAAHQALQHPYFQEQRKTEKRALGSHRKAGFPEHPVAPEPLSNSCQISKEGRKQKQSLKQEEDRPKRRGPAYVMELPKLKLSGVVRLSSYSSPTLQSVLGSGTNGRVPVLRPLKCIPASKKTDPQKDLKPAPQQCRLPTIVRKGGR",
# STK16_HUMAN
"MGHALCVCSRGTVIIDNKRYLFIQKLGEGGFSYVDLVEGLHDGHFYALKRILCHEQQDREEAQREADMHRLFNHPNILRLVAYCLRERGAKHEAWLLLPFFKRGTLWNEIERLKDKGNFLTEDQILWLLLGICRGLEAIHAKGYAHRDLKPTNILLGDEGQPVLMDLGSMNQACIHVEGSRQALTLQDWAAQRCTISYRAPELFSVQSHCVIDERTDVWSLGCVLYAMMFGEGPYDMVFQKGDSVALAVQNQLSIPQSPRHSSALRQLLNSMMTVDPHQRPHIPLLLSQLEALQPPAPGQHTTQI",
# SRPK3_HUMAN
"MSASTGGGGDSGGSGGSSSSSQASCGPESSGSELALATPVPQMLQGLLGSDDEEQEDPKDYCKGGYHPVKIGDVFNGRYHVVRKLGWGHFSTVWLCWDIQRKRFVALKVVKSAGHYTETAVDEIKLLKCVRDSDPSDPKRETIVQLIDDFRISGVNGVHVCMVLEVLGHQLLKWIIKSNYQGLPVPCVKSIVRQVLHGLDYLHTKCKIIHTDIKPENILLCVGDAYIRRLAAEATEWQQAGAPPPSRSIVSTAPQEVLQTGKLSKNKRKKMRRKRKQQKRLLEERLRDLQRLEAMEAATQAEDSGLRLDGGSGSTSSSGCHPGGARAGPSPASSSPAPGGGRSLSAGSQTSGFSGSLFSPASCSILSGSSNQRETGGLLSPSTPFGASNLLVNPLEPQNADKIKIKIADLGNACWVHKHFTEDIQTRQYRAVEVLIGAEYGPPADIWSTACMAFELATGDYLFEPHSGEDYSRDEDHIAHIVELLGDIPPAFALSGRYSREFFNRRGELRHIHNLKHWGLYEVLMEKYEWPLEQATQFSAFLLPMMEYIPEKRASAADCLQHPWLNP",
# WNK1_HUMAN
"MSGGAAEKQSSTPGSLFLSPPAPAPKNGSSSDSSVGEKLGAAAADAVTGRTEEYRRRRHTMDKDSRGAAATTTTTEHRFFRRSVICDSNATALELPGLPLSLPQPSIPAAVPQSAPPEPHREETVTATATSQVAQQPPAAAAPGEQAVAGPAPSTVPSSTSKDRPVSQPSLVGSKEEPPPARSGSGGGSAKEPQEERSQQQDDIEELETKAVGMSNDGRFLKFDIEIGRGSFKTVYKGLDTETTVEVAWCELQDRKLTKSERQRFKEEAEMLKGLQHPNIVRFYDSWESTVKGKKCIVLVTELMTSGTLKTYLKRFKVMKIKVLRSWCRQILKGLQFLHTRTPPIIHRDLKCDNIFITGPTGSVKIGDLGLATLKRASFAKSVIGTPEFMAPEMYEEKYDESVDVYAFGMCMLEMATSEYPYSECQNAAQIYRRVTSGVKPASFDKVAIPEVKEIIEGCIRQNKDERYSIKDLLNHAFFQEETGVRVELAEEDDGEKIAIKLWLRIEDIKKLKGKYKDNEAIEFSFDLERDVPEDVAQEMVESGYVCEGDHKTMAKAIKDRVSLIKRKREQRQLVREEQEKKKQEESSLKQQVEQSSASQTGIKQLPSASTGIPTASTTSASVSTQVEPEEPEADQHQQLQYQQPSISVLSDGTVDSGQGSSVFTESRVSSQQTVSYGSQHEQAHSTGTVPGHIPSTVQAQSQPHGVYPPSSVAQGQSQGQPSSSSLTGVSSSQPIQHPQQQQGIQQTAPPQQTVQYSLSQTSTSSEATTAQPVSQPQAPQVLPQVSAGKQLPVSQPVPTIQGEPQIPVATQPSVVPVHSGAHFLPVGQPLPTPLLPQYPVSQIPISTPHVSTAQTGFSSLPITMAAGITQPLLTLASSATTAAIPGVSTVVPSQLPTLLQPVTQLPSQVHPQLLQPAVQSMGIPANLGQAAEVPLSSGDVLYQGFPPRLPPQYPGDSNIAPSSNVASVCIHSTVLSPPMPTEVLATPGYFPTVVQPYVESNLLVPMGGVGGQVQVSQPGGSLAQAPTTSSQQAVLESTQGVSQVAPAEPVAVAQTQATQPTTLASSVDSAHSDVASGMSDGNENVPSSSGRHEGRTTKRHYRKSVRSRSRHEKTSRPKLRILNVSNKGDRVVECQLETHNRKMVTFKFDLDGDNPEEIATIMVNNDFILAIERESFVDQVREIIEKADEMLSEDVSVEPEGDQGLESLQGKDDYGFSGSQKLEGEFKQPIPASSMPQQIGIPTSSLTQVVHSAGRRFIVSPVPESRLRESKVFPSEITDTVAASTAQSPGMNLSHSASSLSLQQAFSELRRAQMTEGPNTAPPNFSHTGPTFPVVPPFLSSIAGVPTTAAATAPVPATSSPPNDISTSVIQSEVTVPTEEGIAGVATSTGVVTSGGLPIPPVSESPVLSSVVSSITIPAVVSISTTSPSLQVPTSTSEIVVSSTALYPSVTVSATSASAGGSTATPGPKPPAVVSQQAAGSTTVGATLTSVSTTTSFPSTASQLCIQLSSSTSTPTLAETVVVSAHSLDKTSHSSTTGLAFSLSAPSSSSSPGAGVSSYISQPGGLHPLVIPSVIASTPILPQAAGPTSTPLLPQVPSIPPLVQPVANVPAVQQTLIHSQPQPALLPNQPHTHCPEVDSDTQPKAPGIDDIKTLEEKLRSLFSEHSSSGAQHASVSLETSLVIESTVTPGIPTTAVAPSKLLTSTTSTCLPPTNLPLGTVALPVTPVVTPGQVSTPVSTTTSGVKPGTAPSKPPLTKAPVLPVGTELPAGTLPSEQLPPFPGPSLTQSQQPLEDLDAQLRRTLSPEMITVTSAVGPVSMAAPTAITEAGTQPQKGVSQVKEGPVLATSSGAGVFKMGRFQVSVAADGAQKEGKNKSEDAKSVHFESSTSESSVLSSSSPESTLVKPEPNGITIPGISSDVPESAHKTTASEAKSDTGQPTKVGRFQVTTTANKVGRFSVSKTEDKITDTKKEGPVASPPFMDLEQAVLPAVIPKKEKPELSEPSHLNGPSSDPEAAFLSRDVDDGSGSPHSPHQLSSKSLPSQNLSQSLSNSFNSSYMSSDNESDIEDEDLKLELRRLRDKHLKEIQDLQSRQKHEIESLYTKLGKVPPAVIIPPAAPLSGRRRRPTKSKGSKSSRSSSLGNKSPQLSGNLSGQSAASVLHPQQTLHPPGNIPESGQNQLLQPLKPSPSSDNLYSAFTSDGAISVPSLSAPGQGTSSTNTVGATVNSQAAQAQPPAMTSSRKGTFTDDLHKLVDNWARDAMNLSGRRGSKGHMNYEGPGMARKFSAPGQLCISMTSNLGGSAPISAASATSLGHFTKSMCPPQQYGFPATPFGAQWSGTGGPAPQPLGQFQPVGTASLQNFNISNLQKSISNPPGSNLRTT",
# CDKL3_HUMAN
"MEMYETLGKVGEGSYGTVMKCKHKNTGQIVAIKIFYERPEQSVNKIAMREIKFLKQFHHENLVNLIEVFRQKKKIHLVFEFIDHTVLDELQHYCHGLESKRLRKYLFQILRAIDYLHSNNIIHRDIKPENILVSQSGITKLCDFGFARTLAAPGDIYTDYVATRWYRAPELVLKDTSYGKPVDIWALGCMIIEMATGNPYLPSSSDLDLLHKIVLKVGNLSPHLQNIFSKSPIFAGVVLPQVQHPKNARKKYPKLNGLLADIVHACLQIDPADRISSSDLLHHEYFTRDGFIEKFMPELKAKLLQEAKVNSLIKPKESSKENELRKDERKTVYTNTLLSSSVLGKEIEKEKKPKEIKVRVIKVKGGRGDISEPKKKEYEGGLGQQDANENVHPMSPDTKLVTIEPPNPINPSTNCNGLKENPHCGGSVTMPPINLTNSNLMAANLSSNLFHPSVRLTERAKKRRTSSQSIGQVMPNSRQEDPGPIQSQMEKGIFNERTGHSDQMANENKRKLNFSRSDRKEFHFPELPVTIQSKDTKGMEVKQIKMLKRESKKTESSKIPTLLNVDQNQEKQEGGDGHCEGKNLKRNRFFFW",
# PAK5_HUMAN
"MFGKKKKKIEISGPSNFEHRVHTGFDPQEQKFTGLPQQWHSLLADTANRPKPMVDPSCITPIQLAPMKTIVRGNKPCKETSINGLLEDFDNISVTRSNSLRKESPPTPDQGASSHGPGHAEENGFITFSQYSSESDTTADYTTEKYREKSLYGDDLDPYYRGSHAAKQNGHVMKMKHGEAYYSEVKPLKSDFARFSADYHSHLDSLSKPSEYSDLKWEYQRASSSSPLDYSFQFTPSRTAGTSGCSKESLAYSESEWGPSLDDYDRRPKSSYLNQTSPQPTMRQRSRSGSGLQEPMMPFGASAFKTHPQGHSYNSYTYPRLSEPTMCIPKVDYDRAQMVLSPPLSGSDTYPRGPAKLPQSQSKSGYSSSSHQYPSGYHKATLYHHPSLQSSSQYISTASYLSSLSLSSSTYPPPSWGSSSDQQPSRVSHEQFRAALQLVVSPGDPREYLANFIKIGEGSTGIVCIATEKHTGKQVAVKKMDLRKQQRRELLFNEVVIMRDYHHDNVVDMYSSYLVGDELWVVMEFLEGGALTDIVTHTRMNEEQIATVCLSVLRALSYLHNQGVIHRDIKSDSILLTSDGRIKLSDFGFCAQVSKEVPKRKSLVGTPYWMAPEVISRLPYGTEVDIWSLGIMVIEMIDGEPPYFNEPPLQAMRRIRDSLPPRVKDLHKVSSVLRGFLDLMLVREPSQRATAQELLGHPFLKLAGPPSCIVPLMRQYRHH",
# PKN3_HUMAN
"MEEGAPRQPGPSQWPPEDEKEVIRRAIQKELKIKEGVENLRRVATDRRHLGHVQQLLRSSNRRLEQLHGELRELHARILLPGPGPGPAEPVASGPRPWAEQLRARHLEALRRQLHVELKVKQGAENMTHTCASGTPKERKLLAAAQQMLRDSQLKVALLRMKISSLEASGSPEPGPELLAEELQHRLHVEAAVAEGAKNVVKLLSSRRTQDRKALAEAQAQLQESSQKLDLLRLALEQLLEQLPPAHPLRSRVTRELRAAVPGYPQPSGTPVKPTALTGTLQVRLLGCEQLLTAVPGRSPAAALASSPSEGWLRTKAKHQRGRGELASEVLAVLKVDNRVVGQTGWGQVAEQSWDQTFVIPLERARELEIGVHWRDWRQLCGVAFLRLEDFLDNACHQLSLSLVPQGLLFAQVTFCDPVIERRPRLQRQERIFSKRRGQDFLRASQMNLGMAAWGRLVMNLLPPCSSPSTISPPKGCPRTPTTLREASDPATPSNFLPKKTPLGEEMTPPPKPPRLYLPQEPTSEETPRTKRPHMEPRTRRGPSPPASPTRKPPRLQDFRCLAVLGRGHFGKVLLVQFKGTGKYYAIKALKKQEVLSRDEIESLYCEKRILEAVGCTGHPFLLSLLACFQTSSHACFVTEFVPGGDLMMQIHEDVFPEPQARFYVACVVLGLQFLHEKKIIYRDLKLDNLLLDAQGFLKIADFGLCKEGIGFGDRTSTFCGTPEFLAPEVLTQEAYTRAVDWWGLGVLLYEMLVGECPFPGDTEEEVFDCIVNMDAPYPGFLSVQGLEFIQKLLQKCPEKRLGAGEQDAEEIKVQPFFRTTNWQALLARTIQPPFVPTLCGPADLRYFEGEFTGLPPALTPPAPHSLLTARQQAAFRDFDFVSERFLEP",
# SIK2_HUMAN
"MVMADGPRHLQRGPVRVGFYDIEGTLGKGNFAVVKLGRHRITKTEVAIKIIDKSQLDAVNLEKIYREVQIMKMLDHPHIIKLYQVMETKSMLYLVTEYAKNGEIFDYLANHGRLNESEARRKFWQILSAVDYCHGRKIVHRDLKAENLLLDNNMNIKIADFGFGNFFKSGELLATWCGSPPYAAPEVFEGQQYEGPQLDIWSMGVVLYVLVCGALPFDGPTLPILRQRVLEGRFRIPYFMSEDCEHLIRRMLVLDPSKRLTIAQIKEHKWMLIEVPVQRPVLYPQEQENEPSIGEFNEQVLRLMHSLGIDQQKTIESLQNKSYNHFAAIYFLLVERLKSHRSSFPVEQRLDGRQRRPSTIAEQTVAKAQTVGLPVTMHSPNMRLLRSALLPQASNVEAFSFPASGCQAEAAFMEEECVDTPKVNGCLLDPVPPVLVRKGCQSLPSNMMETSIDEGLETEGEAEEDPAHAFEAFQSTRSGQRRHTLSEVTNQLVVMPGAGKIFSMNDSPSLDSVDSEYDMGSVQRDLNFLEDNPSLKDIMLANQPSPRMTSPFISLRPTNPAMQALSSQKREVHNRSPVSFREGRRASDTSLTQGIVAFRQHLQNLARTKGILELNKVQLLYEQIGPEADPNLAPAAPQLQDLASSCPQEEVSQQQESVSTLPASVHPQLSPRQSLETQYLQHRLQKPSLLSKAQNTCQLYCKEPPRSLEQQLQEHRLQQKRLFLQKQSQLQAYFNQMQIAESSYPQPSQQLPLPRQETPPPSQQAPPFSLTQPLSPVLEPSSEQMQYSPFLSQYQEMQLQPLPSTSGPRAAPPLPTQLQQQQPPPPPPPPPPRQPGAAPAPLQFSYQTCELPSAASPAPDYPTPCQYPVDGAQQSDLTGPDCPRSPGLQEAPSSYDPLALSELPGLFDCEMLDAVDPQHNGYVLVN",
# MARK4_HUMAN
"MSSRTVLAPGNDRNSDTHGTLGSGRSSDKGPSWSSRSLGARCRNSIASCPEEQPHVGNYRLLRTIGKGNFAKVKLARHILTGREVAIKIIDKTQLNPSSLQKLFREVRIMKGLNHPNIVKLFEVIETEKTLYLVMEYASAGEVFDYLVSHGRMKEKEARAKFRQIVSAVHYCHQKNIVHRDLKAENLLLDAEANIKIADFGFSNEFTLGSKLDTFCGSPPYAAPELFQGKKYDGPEVDIWSLGVILYTLVSGSLPFDGHNLKELRERVLRGKYRVPFYMSTDCESILRRFLVLNPAKRCTLEQIMKDKWINIGYEGEELKPYTEPEEDFGDTKRIEVMVGMGYTREEIKESLTSQKYNEVTATYLLLGRKTEEGGDRGAPGLALARVRAPSDTTNGTSSSKGTSHSKGQRSSSSTYHRQRRHSDFCGPSPAPLHPKRSPTSTGEAELKEERLPGRKASCSTAGSGSRGLPPSSPMVSSAHNPNKAEIPERRKDSTSTPNNLPPSMMTRRNTYVCTERPGAERPSLLPNGKENSSGTPRVPPASPSSHSLAPPSGERSRLARGSTIRSTFHGGQVRDRRAGGGGGGGVQNGPPASPTLAHEAAPLPAGRPRPTTNLFTKLTSKLTRRVADEPERIGGPEVTSCHLPWDQTETAPRLLRFPWSVKLTSSRPPEALMAALRQATAAARCRCRQPQPFLLACLHGGAGGPEPLSHFEVEVCQLPRPGLRGVLFRRVAGTALAFRTLVTRISNDLEL",
# DUSTY_HUMAN
"MEGDGVPWGSEPVSGPGPGGGGMIRELCRGFGRYRRYLGRLRQNLRETQKFFRDIKCSHNHTCLSSLTGGGGAERGPAGDVAETGLQAGQLSCISFPPKEEKYLQQIVDCLPCILILGQDCNVKCQLLNLLLGVQVLPTTKLGSEESCKLRRLRFTYGTQTRVSLALPGQYELVHTLVAHQGNWETIPEEDLEVQENNEDAAHVLAELEVTMHHALLQEVDVVVAPCQGLRPTVDVLGDLVNDFLPVITYALHKDELSERDEQELQEIRKYFSFPVFFFKVPKLGSEIIDSSTRRMESERSPLYRQLIDLGYLSSSHWNCGAPGQDTKAQSMLVEQSEKLRHLSTFSHQVLQTRLVDAAKALNLVHCHCLDIFINQAFDMQRDLQITPKRLEYTRKKENELYESLMNIANRKQEEMKDMIVETLNTMKEELLDDATNMEFKDVIVPENGEPVGTREIKCCIRQIQELIISRLNQAVANKLISSVDYLRESFVGTLERCLQSLEKSQDVSVHITSNYLKQILNAAYHVEVTFHSGSSVTRMLWEQIKQIIQRITWVSPPAITLEWKRKVAQEAIESLSASKLAKSICSQFRTRLNSSHEAFAASLRQLEAGHSGRLEKTEDLWLRVRKDHAPRLARLSLESCSLQDVLLHRKPKLGQELGRGQYGVVYLCDNWGGHFPCALKSVVPPDEKHWNDLALEFHYMRSLPKHERLVDLHGSVIDYNYGGGSSIAVLLIMERLHRDLYTGLKAGLTLETRLQIALDVVEGIRFLHSQGLVHRDIKLKNVLLDKQNRAKITDLGFCKPEAMMSGSIVGTPIHMAPELFTGKYDNSVDVYAFGILFWYICSGSVKLPEAFERCASKDHLWNNVRRGARPERLPVFDEECWQLMEACWDGDPLKRPLLGIVQPMLQGIMNRLCKSNSEQPNRGLDDST",
# KS6C1_HUMAN
"MTSYRERSADLARFYTVTEPQRHPRGYTVYKVTARVVSRRNPEDVQEIIVWKRYSDFKKLHKELWQIHKNLFRHSELFPPFAKGIVFGRFDETVIEERRQCAEDLLQFSANIPALYNSKQLEDFFKGGIINDSSELIGPAEAHSDSLIDTFPECSTEGFSSDSDLVSLTVDVDSLAELDDGMASNQNSPIRTFGLNLSSDSSALGAVASDSEQSKTEEERESRSLFPGSLKPKLGKRDYLEKAGELIKLALKKEEEDDYEAASDFYRKGVDLLLEGVQGESSPTRREAVKRRTAEYLMRAESISSLYGKPQLDDVSQPPGSLSSRPLWNLRSPAEELKAFRVLGVIDKVLLVMDTRTEQTFILKGLRKSSEYSRNRKTIIPRCVPNMVCLHKYIISEESVFLVLQHAEGGKLWSYISKFLNRSPEESFDIKEVKKPTLAKVHLQQPTSSPQDSSSFESRGSDGGSMLKALPLKSSLTPSSQDDSNQEDDGQDSSPKWPDSGSSSEEECTTSYLTLCNEYGQEKIEPGSLNEEPFMKTEGNGVDTKAIKSFPAHLAADSDSPSTQLRAHELKFFPNDDPEAVSSPRTSDSLSRSKNSPMEFFRIDSKDSASELLGLDFGEKLYSLKSEPLKPFFTLPDGDSASRSFNTSESKVEFKAQDTISRGSDDSVPVISFKDAAFDDVSGTDEGRPDLLVNLPGELESTREAAAMGPTKFTQTNIGIIENKLLEAPDVLCLRLSTEQCQAHEEKGIEELSDPSGPKSYSITEKHYAQEDPRMLFVAAVDHSSSGDMSLLPSSDPKFQGLGVVESAVTANNTEESLFRICSPLSGANEYIASTDTLKTEEVLLFTDQTDDLAKEEPTSLFQRDSETKGESGLVLEGDKEIHQIFEDLDKKLALASRFYIPEGCIQRWAAEMVVALDALHREGIVCRDLNPNNILLNDRGHIQLTYFSRWSEVEDSCDSDAIERMYCAPEVGAITEETEACDWWSLGAVLFELLTGKTLVECHPAGINTHTTLNMPECVSEEARSLIQQLLQFNPLERLGAGVAGVEDIKSHPFFTPVDWAELMR",
# KS6A6_HUMAN
"MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSCHDEGVVKEIPITHHVKEGYEKADPAQFELLKVLGQGSFGKVFLVRKKTGPDAGQLYAMKVLKKASLKVRDRVRTKMERDILVEVNHPFIVKLHYAFQTEGKLYLILDFLRGGDVFTRLSKEVLFTEEDVKFYLAELALALDHLHQLGIVYRDLKPENILLDEIGHIKLTDFGLSKESVDQEKKAYSFCGTVEYMAPEVVNRRGHSQSADWWSYGVLMFEMLTGTLPFQGKDRNETMNMILKAKLGMPQFLSAEAQSLLRMLFKRNPANRLGSEGVEEIKRHLFFANIDWDKLYKREVQPPFKPASGKPDDTFCFDPEFTAKTPKDSPGLPASANAHQLFKGFSFVATSIAEEYKITPITSANVLPIVQINGNAAQFGEVYELKEDIGVGSYSVCKRCIHATTNMEFAVKIIDKSKRDPSEEIEILMRYGQHPNIITLKDVFDDGRYVYLVTDLMKGGELLDRILKQKCFSEREASDILYVISKTVDYLHCQGVVHRDLKPSNILYMDESASADSIRICDFGFAKQLRGENGLLLTPCYTANFVAPEVLMQQGYDAACDIWSLGVLFYTMLAGYTPFANGPNDTPEEILLRIGNGKFSLSGGNWDNISDGAKDLLSHMLHMDPHQRYTAEQILKHSWITHRDQLPNDQPKRNDVSHVVKGAMVATYSALTHKTFQPVLEPVAASSLAQRRSMKKRTSTGL",
# NRBP2_HUMAN
"MAAPEPAPRRAREREREREDESEDESDILEESPCGRWQKRREQVNQGNMPGLQSTFLAMDTEEGVEVVWNELHFGDRKAFAAHEEKIQTVFEQLVLVDHPNIVKLHKYWLDTSEACARVIFITEYVSSGSLKQFLKKTKKNHKAMNARAWKRWCTQILSALSFLHACSPPIIHGNLTSDTIFIQHNGLIKIGSVWHRIFSNALPDDLRSPIRAEREELRNLHFFPPEYGEVADGTAVDIFSFGMCALEMAVLEIQTNGDTRVTEEAIARARHSLSDPNMREFILCCLARDPARRPSAHSLLFHRVLFEVHSLKLLAAHCFIQHQYLMPENVVEEKTKAMDLHAVLAELPRPRRPPLQWRYSEVSFMELDKFLEDVRNGIYPLMNFAATRPLGLPRVLAPPPEEVQKAKTPTPEPFDSETRKVIQMQCNLERSEDKARWHLTLLLVLEDRLHRQLTYDLLPTDSAQDLASELVHYGFLHEDDRMKLAAFLESTFLKYRGTQA",
# STKL1_HUMAN
"MLGPGSNRRRPTQGERGPGSPGEPMEKYQVLYQLNPGALGVNLVVEEMETKVKHVIKQVECMDDHYASQALEELMPLLKLRHAHISVYQELFITWNGEISSLYLCLVMEFNELSFQEVIEDKRKAKKIIDSEWMQNVLGQVLDALEYLHHLDIIHRNLKPSNIILISSDHCKLQDLSSNVLMTDKAKWNIRAEEDPFRKSWMAPEALNFSFSQKSDIWSLGCIILDMTSCSFMDGTEAMHLRKSLRQSPGSLKAVLKTMEEKQIPDVETFRNLLPLMLQIDPSDRITIKDVVHITFLRGSFKSSCVSLTLHRQMVPASITDMLLEGNVASILEVMQKFSGWPEVQLRAMKRLLKMPADQLGLPWPPELVEVVVTTMELHDRVLDVQLCACSLLLHLLGQALVHHPEAKAPCNQAITSTLLSALQSHPEEEPLLVMVYSLLAITTTQESESLSEELQNAGLLEHILEHLNSSLESRDVCASGLGLLWALLLDGIIVNKAPLEKVPDLISQVLATYPADGEMAEASCGVFWLLSLLGCIKEQQFEQVVALLLQSIRLCQDRALLVNNAYRGLASLVKVSELAAFKVVVQEEGGSGLSLIKETYQLHRDDPEVVENVGMLLVHLASYEEILPELVSSSMKALLQEIKERFTSSLVSDSSAFSKPGLPPGGSPQLGCTTSGGLE",
# SGK2_HUMAN
"MNSSPAGTPSPQPSRANGNINLGPSANPNAQPTDFDFLKVIGKGNYGKVLLAKRKSDGAFYAVKVLQKKSILKKKEQSHIMAERSVLLKNVRHPFLVGLRYSFQTPEKLYFVLDYVNGGELFFHLQRERRFLEPRARFYAAEVASAIGYLHSLNIIYRDLKPENILLDCQGHVVLTDFGLCKEGVEPEDTTSTFCGTPEYLAPEVLRKEPYDRAVDWWCLGAVLYEMLHGLPPFYSQDVSQMYENILHQPLQIPGGRTVAACDLLQSLLHKDQRQRLGSKADFLEIKNHVFFSPINWDDLYHKRLTPPFNPNVTGPADLKHFDPEFTQEAVSKSIGCTPDTVASSSGASSAFLGFSYAPEDDDILDC",
# TSSK6_HUMAN
"MSGDKLLSELGYKLGRTIGEGSYSKVKVATSKKYKGTVAIKVVDRRRAPPDFVNKFLPRELSILRGVRHPHIVHVFEFIEVCNGKLYIVMEAAATDLLQAVQRNGRIPGVQARDLFAQIAGAVRYLHDHHLVHRDLKCENVLLSPDERRVKLTDFGFGRQAHGYPDLSTTYCGSAAYASPEVLLGIPYDPKKYDVWSMGVVLYVMVTGCMPFDDSDIAGLPRRQKRGVLYPEGLELSERCKALIAELLQFSPSARPSAGQVARNCWLRAGDSG",
# SGK3_HUMAN
"MQRDHTMDYKESCPSVSIPSSDEHREKKKRFTVYKVLVSVGRSEWFVFRRYAEFDKLYNTLKKQFPAMALKIPAKRIFGDNFDPDFIKQRRAGLNEFIQNLVRYPELYNHPDVRAFLQMDSPKHQSDPSEDEDERSSQKLHSTSQNINLGPSGNPHAKPTDFDFLKVIGKGSFGKVLLAKRKLDGKFYAVKVLQKKIVLNRKEQKHIMAERNVLLKNVKHPFLVGLHYSFQTTEKLYFVLDFVNGGELFFHLQRERSFPEHRARFYAAEIASALGYLHSIKIVYRDLKPENILLDSVGHVVLTDFGLCKEGIAISDTTTTFCGTPEYLAPEVIRKQPYDNTVDWWCLGAVLYEMLYGLPPFYCRDVAEMYDNILHKPLSLRPGVSLTAWSILEELLEKDRQNRLGAKEDFLEIQNHPFFESLSWADLVQKKIPPPFNPNVAGPDDIRNFDTAFTEETVPYSVCVSSDYSIVNASVLEADDAFVGFSYAPPSEDLFL",
# TTBK1_HUMAN
"MQCLAAALKDETNMSGGGEQADILPANYVVKDRWKVLKKIGGGGFGEIYEAMDLLTRENVALKVESAQQPKQVLKMEVAVLKKLQGKDHVCRFIGCGRNEKFNYVVMQLQGRNLADLRRSQPRGTFTLSTTLRLGKQILESIEAIHSVGFLHRDIKPSNFAMGRLPSTYRKCYMLDFGLARQYTNTTGDVRPPRNVAGFRGTVRYASVNAHKNREMGRHDDLWSLFYMLVEFAVGQLPWRKIKDKEQVGMIKEKYEHRMLLKHMPSEFHLFLDHIASLDYFTKPDYQLIMSVFENSMKERGIAENEAFDWEKAGTDALLSTSTSTPPQQNTRQTAAMFGVVNVTPVPGDLLRENTEDVLQGEHLSDQENAPPILPGRPSEGLGPSPHLVPHPGGPEAEVWEETDVNRNKLRINIGKSPCVEEEQSRGMGVPSSPVRAPPDSPTTPVRSLRYRRVNSPESERLSTADGRVELPERRSRMDLPGSPSRQACSSQPAQMLSVDTGHADRQASGRMDVSASVEQEALSNAFRSVPLAEEEDFDSKEWVIIDKETELKDFPPGAEPSTSGTTDEEPEELRPLPEEGEERRRLGAEPTVRPRGRSMQALAEEDLQHLPPQPLPPQLSQGDGRSETSQPPTPGSPSHSPLHSGPRPRRRESDPTGPQRQVFSVAPPFEVNGLPRAVPLSLPYQDFKRDLSDYRERARLLNRVRRVGFSHMLLTTPQVPLAPVQPQANGKEEEEEEEEDEEEEEEDEEEEEEEEEEEEEEEEEEEEEEEAAAAVALGEVLGPRSGSSSEGSERSTDRSQEGAPSTLLADDQKESRGRASMADGDLEPEEGSKTLVLVSPGDMKKSPVTAELAPDPDLGTLAALTPQHERPQPTGSQLDVSEPGTLSSVLKSEPKPPGPGAGLGAGTVTTGVGGVAVTSSPFTKVERTFVHIAEKTHLNVMSSGGQALRSEEFSAGGELGLELASDGGAVEEGARAPLENGLALSGLNGAEIEGSALSGAPRETPSEMATNSLPNGPALADGPAPVSPLEPSPEKVATISPRRHAMPGSRPRSRIPVLLSEEDTGSEPSGSLSAKERWSKRARPQQDLARLVMEKRQGRLLLRLASGASSSSSEEQRRASETLSGTGSEEDTPASEPAAALPRKSGRAAATRSRIPRPIGLRMPMPVAAQQPASRSHGAAPALDTAITSRLQLQTPPGSATAADLRPKQPPGRGLGPGRAQAGARPPAPRSPRLPASTSAARNASASPRSQSLSRRESPSPSHQARPGVPPPRGVPPARAQPDGTPSPGGSKKGPRGKLQAQRATTKGRAGGAEGRAGAR",
# DCLK2_HUMAN
"MASTRSIELEHFEERDKRPRPGSRRGAPSSSGGSSSSGPKGNGLIPSPAHSAHCSFYRTRTLQALSSEKKAKKARFYRNGDRYFKGLVFAISSDRFRSFDALLIELTRSLSDNVNLPQGVRTIYTIDGSRKVTSLDELLEGESYVCASNEPFRKVDYTKNINPNWSVNIKGGTSRALAAASSVKSEVKESKDFIKPKLVTVIRSGVKPRKAVRILLNKKTAHSFEQVLTDITEAIKLDSGVVKRLCTLDGKQVTCLQDFFGDDDVFIACGPEKFRYAQDDFVLDHSECRVLKSSYSRSSAVKYSGSKSPGPSRRSKSPASVNGTPSSQLSTPKSTKSSSSSPTSPGSFRGLKQISAHGRSSSNVNGGPELDRCISPEGVNGNRCSESSTLLEKYKIGKVIGDGNFAVVKECIDRSTGKEFALKIIDKAKCCGKEHLIENEVSILRRVKHPNIIMLVEEMETATELFLVMELVKGGDLFDAITSSTKYTERDGSAMVYNLANALRYLHGLSIVHRDIKPENLLVCEYPDGTKSLKLGDFGLATVVEGPLYTVCGTPTYVAPEIIAETGYGLKVDIWAAGVITYILLCGFPPFRSENNLQEDLFDQILAGKLEFPAPYWDNITDSAKELISQMLQVNVEARCTAGQILSHPWVSDDASQENNMQAEVTGKLKQHFNNALPKQNSTTTGVSVIMNTALDKEGQIFCSKHCQDSGRPGMEPISPVPPSVEEIPVPGEAVPAPTPPESPTPHPPPAAPGGERAGTWRRHRD",
# PXK_HUMAN
"MAFMEKPPAGKVLLDDTVPLTAAIEASQSLQSHTEYIIRVQRGISVENSWQIVRRYSDFDLLNNSLQIAGLSLPLPPKKLIGNMDREFIAERQKGLQNYLNVITTNHILSNCELVKKFLDPNNYSANYTEIALQQVSMFFRSEPKWEVVEPLKDIGWRIRKKYFLMKIKNQPKERLVLSWADLGPDKYLSDKDFQCLIKLLPSCLHPYIYRVTFATANESSALLIRMFNEKGTLKDLIYKAKPKDPFLKKYCNPKKIQGLELQQIKTYGRQILEVLKFLHDKGFPYGHLHASNVMLDGDTCRLLDLENSLLGLPSFYRSYFSQFRKINTLESVDVHCFGHLLYEMTYGRPPDSVPVDSFPPAPSMAVVAVLESTLSCEACKNGMPTISRLLQMPLFSDVLLTTSEKPQFKIPTKLKEALRIAKECIEKRLIEEQKQIHQHRRLTRAQSHHGSEEERKKRKILARKKSKRSALENSEEHSAKYSNSNNSAGSGASSPLTSPSSPTPPSTSGISALPPPPPPPPPPAAPLPPASTEAPAQLSSQAVNGMSRGALLSSIQNFQKGTLRKAKTCDHSAPKIG",
# TOPK_HUMAN
"MEGISNFKTPSKLSEKKKSVLCSTPTINIPASPFMQKLGFGTGVNVYLMKRSPRGLSHSPWAVKKINPICNDHYRSVYQKRLMDEAKILKSLHHPNIVGYRAFTEANDGSLCLAMEYGGEKSLNDLIEERYKASQDPFPAAIILKVALNMARGLKYLHQEKKLLHGDIKSSNVVIKGDFETIKICDVGVSLPLDENMTVTDPEACYIGTEPWKPKEAVEENGVITDKADIFAFGLTLWEMMTLSIPHINLSNDDDDEDKTFDESDFDDEAYYAALGTRPPINMEELDESYQKVIELFSVCTNEDPKDRPSAAHIVEALETDV",
# STYK1_HUMAN
"MGMTRMLLECSLSDKLCVIQEKQYEVIIVPTLLVTIFLILLGVILWLFIREQRTQQQRSGPQGIAPVPPPRDLSWEAGHGGNVALPLKETSVENFLGATTPALAKLQVPREQLSEVLEQICSGSCGPIFRANMNTGDPSKPKSVILKALKEPAGLHEVQDFLGRIQFHQYLGKHKNLVQLEGCCTEKLPLYMVLEDVAQGDLLSFLWTCRRDVMTMDGLLYDLTEKQVYHIGKQVLLALEFLQEKHLFHGDVAARNILMQSDLTAKLCGLGLAYEVYTRGAISSTQTIPLKWLAPERLLLRPASIRADVWSFGILLYEMVTLGAPPYPEVPPTSILEHLQRRKIMKRPSSCTHTMYSIMKSCWRWREADRPSPRELRLRLEAAIKTADDEAVLQVPELVVPELYAAVAGIRVESLFYNYSML",
# TBK1_HUMAN
"MQSTSNHLWLLSDILGQGATANVFRGRHKKTGDLFAIKVFNNISFLRPVDVQMREFEVLKKLNHKNIVKLFAIEEETTTRHKVLIMEFCPCGSLYTVLEEPSNAYGLPESEFLIVLRDVVGGMNHLRENGIVHRDIKPGNIMRVIGEDGQSVYKLTDFGAARELEDDEQFVSLYGTEEYLHPDMYERAVLRKDHQKKYGATVDLWSIGVTFYHAATGSLPFRPFEGPRRNKEVMYKIITGKPSGAISGVQKAENGPIDWSGDMPVSCSLSRGLQVLLTPVLANILEADQEKCWGFDQFFAETSDILHRMVIHVFSLQQMTAHKIYIHSYNTATIFHELVYKQTKIISSNQELIYEGRRLVLEPGRLAQHFPKTTEENPIFVVSREPLNTIGLIYEKISLPKVHPRYDLDGDASMAKAITGVVCYACRIASTLLLYQELMRKGIRWLIELIKDDYNETVHKKTEVVITLDFCIRNIEKTVKVYEKLMKINLEAAELGEISDIHTKLLRLSSSQGTIETSLQDIDSRLSPGGSLADAWAHQEGTHPKDRNVEKLQVLLNCMTEIYYQFKKDKAERRLAYNEEQIHKFDKQKLYYHATKAMTHFTDECVKKYEAFLNKSEEWIRKMLHLRKQLLSLTNQCFDIEEEVSKYQEYTNELQETLPQKMFTASSGIKHTMTPIYPSSNTLVEMTLGMKKLKEEMEGVVKELAENNHILERFGSLTMDGGLRNVDCL",
# TESK2_HUMAN
"MDRSKRNSIAGFPPRVERLEEFEGGGGGEGNVSQVGRVWPSSYRALISAFSRLTRLDDFTCEKIGSGFFSEVFKVRHRASGQVMALKMNTLSSNRANMLKEVQLMNRLSHPNILRFMGVCVHQGQLHALTEYINSGNLEQLLDSNLHLPWTVRVKLAYDIAVGLSYLHFKGIFHRDLTSKNCLIKRDENGYSAVVADFGLAEKIPDVSMGSEKLAVVGSPFWMAPEVLRDEPYNEKADVFSYGIILCEIIARIQADPDYLPRTENFGLDYDAFQHMVGDCPPDFLQLTFNCCNMDPKLRPSFVEIGKTLEEILSRLQEEEQERDRKLQPTARGLLEKAPGVKRLSSLDDKIPHKSPCPRRTIWLSRSQSDIFSRKPPRTVSVLDPYYRPRDGAARTPKVNPFSARQDLMGGKIKFFDLPSKSVISLVFDLDAPGPGTMPLADWQEPLAPPIRRWRSLPGSPEFLHQEACPFVGREESLSDGPPPRLSSLKYRVKEIPPFRASALPAAQAHEAMDCSILQEENGFGSRPQGTSPCPAGASEEMEVEERPAGSTPATFSTSGIGLQTQGKQDG",
# KALRN_HUMAN
"MTDRFWDQWYLWYLRLLRLLDRGSFRNDGLKASDVLPILKEKVAFVSGGRDKRGGPILTFPARSNHDRIRQEDLRKLVTYLASVPSEDVCKRGFTVIIDMRGSKWDLIKPLLKTLQEAFPAEIHVALIIKPDNFWQKQKTNFGSSKFIFETSMVSVEGLTKLVDPSQLTEEFDGSLDYNHEEWIELRLSLEEFFNSAVHLLSRLEDLQEMLARKEFPVDVEGSRRLIDEHTQLKKKVLKAPVEELDREGQRLLQCIRCSDGFSGRNCIPGSADFQSLVPKITSLLDKLHSTRQHLHQMWHVRKLKLDQCFQLRLFEQDAEKMFDWISHNKELFLQSHTEIGVSYQYALDLQTQHNHFAMNSMNAYVNINRIMSVASRLSEAGHYASQQIKQISTQLDQEWKSFAAALDERSTILAMSAVFHQKAEQFLSGVDAWCKMCSEGGLPSEMQDLELAIHHHQTLYEQVTQAYTEVSQDGKALLDVLQRPLSPGNSESLTATANYSKAVHQVLDVVHEVLHHQRRLESIWQHRKVRLHQRLQLCVFQQDVQQVLDWIENHGEAFLSKHTGVGKSLHRARALQKRHDDFEEVAQNTYTNADKLLEAAEQLAQTGECDPEEIYKAARHLEVRIQDFVRRVEQRKLLLDMSVSFHTHTKELWTWMEDLQKEMLEDVCADSVDAVQELIKQFQQQQTATLDATLNVIKEGEDLIQQLRSAPPSLGEPSEARDSAVSNNKTPHSSSISHIESVLQQLDDAQVQMEELFHERKIKLDIFLQLRIFEQYTIEVTAELDAWNEDLLRQMNDFNTEDLTLAEQRLQRHTERKLAMNNMTFEVIQQGQDLHQYITEVQASGIELICEKDIDLAAQVQELLEFLHEKQHELELNAEQTHKRLEQCLQLRHLQAEVKQVLGWIRNGESMLNASLVNASSLSEAEQLQREHEQFQLAIESLFHATSLQKTHQSALQVQQKAEVLLQAGHYDADAIRECAEKVALHWQQLMLKMEDRLKLVNASVAFYKTSEQVCSVLESLEQEYRRDEDWCGGRDKLGPAAEIDHVIPLISKHLEQKEAFLKACTLARRNAEVFLKYIHRNNVSMPSVASHTRGPEQQVKAILSELLQRENRVLHFWTLKKRRLDQCQQYVVFERSAKQALDWIQETGEFYLSTHTSTGETTEETQELLKEYGEFRVPAKQTKEKVKLLIQLADSFVEKGHIHATEIRKWVTTVDKHYRDFSLRMGKYRYSLEKALGVNTEDNKDLELDIIPASLSDREVKLRDANHEVNEEKRKSARKKEFIMAELLQTEKAYVRDLHECLETYLWEMTSGVEEIPPGILNKEHIIFGNIQEIYDFHNNIFLKELEKYEQLPEDVGHCFVTWADKFQMYVTYCKNKPDSNQLILEHAGTFFDEIQQRHGLANSISSYLIKPVQRITKYQLLLKELLTCCEEGKGELKDGLEVMLSVPKKANDAMHVSMLEGFDENLDVQGELILQDAFQVWDPKSLIRKGRERHLFLFEISLVFSKEIKDSSGHTKYVYKNKLLTSELGVTEHVEGDPCKFALWSGRTPSSDNKTVLKASNIETKQEWIKNIREVIQERIIHLKGALKEPLQLPKTPAKQRNNSKRDGVEDIDSQGDGSSQPDTISIASRTSQNTVDSDKLSGGCELTVVLQDFSAGHSSELTIQVGQTVELLERPSERPGWCLVRTTERSPPLEGLVPSSALCISHSRSSVEMDCFFPLVKDAYSHSSSENGGKSESVANLQAQPSLNSIHSSPGPKRSTNTLKKWLTSPVRRLNSGKADGNIKKQKKVRDGRKSFDLGSPKPGDETTPQGDSADEKSKKGWGEDEPDEESHTPLPPPMKIFDNDPTQDEMSSSLLAARQASTEVPTAADLVNAIEKLVKNKLSLEGSSYRGSLKDPAGCLNEGMAPPTPPKNPEEEQKAKALRGRMFVLNELVQTEKDYVKDLGIVVEGFMKRIEEKGVPEDMRGKDKIVFGNIHQIYDWHKDFFLAELEKCIQEQDRLAQLFIKHERKLHIYVWYCQNKPRSEYIVAEYDAYFEEVKQEINQRLTLSDFLIKPIQRITKYQLLLKDFLRYSEKAGLECSDIEKAVELMCLVPKRCNDMMNLGRLQGFEGTLTAQGKLLQQDTFYVIELDAGMQSRTKERRVFLFEQIVIFSELLRKGSLTPGYMFKRSIKMNYLVLEENVDNDPCKFALMNRETSERVVLQAANADIQQAWVQDINQVLETQRDFLNALQSPIEYQRKERSTAVMRSQPARLPQASPRPYSSVPAGSEKPPKGSSYNPPLPPLKISTSNGSPGFEYHQPGDKFEASKQNDLGGCNGTSSMAVIKDYYALKENEICVSQGEVVQVLAVNQQNMCLVYQPASDHSPAAEGWVPGSILAPLTKATAAESSDGSIKKSCSWHTLRMRKRAEVENTGKNEATGPRKPKDILGNKVSVKETNSSEESECDDLDPNTSMEILNPNFIQEVAPEFLVPLVDVTCLLGDTVILQCKVCGRPKPTITWKGPDQNILDTDNSSATYTVSSCDSGEITLKICNLMPQDSGIYTCIATNDHGTTSTSATVKVQGVPAAPNRPIAQERSCTSVILRWLPPSSTGNCTISGYTVEYREEGSQIWQQSVASTLDTYLVIEDLSPGCPYQFRVSASNPWGISLPSEPSEFVRLPEYDAAADGATISWKENFDSAYTELNEIGRGRFSIVKKCIHKATRKDVAVKFVSKKMKKKEQAAHEAALLQHLQHPQYITLHDTYESPTSYILILELMDDGRLLDYLMNHDELMEEKVAFYIRDIMEALQYLHNCRVAHLDIKPENLLIDLRIPVPRVKLIDLEDAVQISGHFHIHHLLGNPEFAAPEVIQGIPVSLGTDIWSIGVLTYVMLSGVSPFLDESKEETCINVCRVDFSFPHEYFCGVSNAARDFINVILQEDFRRRPTAATCLQHPWLQPHNGSYSKIPLDTSRLACFIERRKHQNDVRPIPNVKSYIVNRVNQGT",
# TSSK4_HUMAN
"MGKGDVLEAAPTTTAYHSLMDEYGYEVGKAIGHGSYGSVYEAFYTKQKVMVAVKIISKKKASDDYLNKFLPREIQVMKVLRHKYLINFYRAIESTSRVYIILELAQGGDVLEWIQRYGACSEPLAGKWFSQLTLGIAYLHSKSIVHRDLKLENLLLDKWENVKISDFGFAKMVPSNQPVGCSPSYRQVNCFSHLSQTYCGSFAYACPEILRGLPYNPFLSDTWSMGVILYTLVVAHLPFDDTNLKKLLRETQKEVTFPANHTISQECKNLILQMLRQATKRATILDIIKDSWVLKFQPEQPTHEIRLLEAMCQLHNTTKQHQSLQITT",
# VRK3_HUMAN
"MISFCPDCGKSIQAAFKFCPYCGNSLPVEEHVGSQTFVNPHVSSFQGSKRGLNSSFETSPKKVKWSSTVTSPRLSLFSDGDSSESEDTLSSSERSKGSGSRPPTPKSSPQKTRKSPQVTRGSPQKTSCSPQKTRQSPQTLKRSRVTTSLEALPTGTVLTDKSGRQWKLKSFQTRDNQGILYEAAPTSTLTCDSGPQKQKFSLKLDAKDGRLFNEQNFFQRAAKPLQVNKWKKLYSTPLLAIPTCMGFGVHQDKYRFLVLPSLGRSLQSALDVSPKHVLSERSVLQVACRLLDALEFLHENEYVHGNVTAENIFVDPEDQSQVTLAGYGFAFRYCPSGKHVAYVEGSRSPHEGDLEFISMDLHKGCGPSRRSDLQSLGYCMLKWLYGFLPWTNCLPNTEDIMKQKQKFVDKPGPFVGPCGHWIRPSETLQKYLKVVMALTYEEKPPYAMLRNNLEALLQDLRVSPYDPIGLPMVP",
# MYLK3_HUMAN
"MSGTSKESLGHGGLPGLGKTCLTTMDTKLNMLNEKVDQLLHFQEDVTEKLQSMCRDMGHLERGLHRLEASRAPGPGGADGVPHIDTQAGWPEVLELVRAMQQDAAQHGARLEALFRMVAAVDRAIALVGATFQKSKVADFLMQGRVPWRRGSPGDSPEENKERVEEEGGKPKHVLSTSGVQSDAREPGEESQKADVLEGTAERLPPIRASGLGADPAQAVVSPGQGDGVPGPAQAFPGHLPLPTKVEAKAPETPSENLRTGLELAPAPGRVNVVSPSLEVAPGAGQGASSSRPDPEPLEEGTRLTPGPGPQCPGPPGLPAQARATHSGGETPPRISIHIQEMDTPGEMLMTGRGSLGPTLTTEAPAAAQPGKQGPPGTGRCLQAPGTEPGEQTPEGARELSPLQESSSPGGVKAEEEQRAGAEPGTRPSLARSDDNDHEVGALGLQQGKSPGAGNPEPEQDCAARAPVRAEAVRRMPPGAEAGSVVLDDSPAPPAPFEHRVVSVKETSISAGYEVCQHEVLGGGRFGQVHRCTEKSTGLPLAAKIIKVKSAKDREDVKNEINIMNQLSHVNLIQLYDAFESKHSCTLVMEYVDGGELFDRITDEKYHLTELDVVLFTRQICEGVHYLHQHYILHLDLKPENILCVNQTGHQIKIIDFGLARRYKPREKLKVNFGTPEFLAPEVVNYEFVSFPTDMWSVGVITYMLLSGLSPFLGETDAETMNFIVNCSWDFDADTFEGLSEEAKDFVSRLLVKEKSCRMSATQCLKHEWLNNLPAKASRSKTRLKSQLLLQKYIAQRKWKKHFYVVTAANRLRKFPTSP",
# SPEG_HUMAN
"MQKARGTRGEDAGTRAPPSPGVPPKRAKVGAGGGAPVAVAGAPVFLRPLKNAAVCAGSDVRLRVVVSGTPQPSLRWFRDGQLLPAPAPEPSCLWLRRCGAQDAGVYSCMAQNERGRASCEAVLTVLEVGDSETAEDDISDVQGTQRLELRDDGAFSTPTGGSDTLVGTSLDTPPTSVTGTSEEQVSWWGSGQTVLEQEAGSGGGTRRLPGSPRQAQATGAGPRHLGVEPLVRASRANLVGASWGSEDSLSVASDLYGSAFSLYRGRALSIHVSVPQSGLRREEPDLQPQLASEAPRRPAQPPPSKSALLPPPSPRVGKRSPPGPPAQPAATPTSPHRRTQEPVLPEDTTTEEKRGKKSKSSGPSLAGTAESRPQTPLSEASGRLSALGRSPRLVRAGSRILDKLQFFEERRRSLERSDSPPAPLRPWVPLRKARSLEQPKSERGAPWGTPGASQEELRAPGSVAERRRLFQQKAASLDERTRQRSPASDLELRFAQELGRIRRSTSREELVRSHESLRATLQRAPSPREPGEPPLFSRPSTPKTSRAVSPAAAQPPSPSSAEKPGDEPGRPRSRGPAGRTEPGEGPQQEVRRRDQFPLTRSRAIQECRSPVPPPAADPPEARTKAPPGRKREPPAQAVRFLPWATPGLEGAAVPQTLEKNRAGPEAEKRLRRGPEEDGPWGPWDRRGARSQGKGRRARPTSPELESSDDSYVSAGEEPLEAPVFEIPLQNVVVAPGADVLLKCIITANPPPQVSWHKDGSALRSEGRLLLRAEGERHTLLLREARAADAGSYMATATNELGQATCAASLTVRPGGSTSPFSSPITSDEEYLSPPEEFPEPGETWPRTPTMKPSPSQNRRSSDTGSKAPPTFKVSLMDQSVREGQDVIMSIRVQGEPKPVVSWLRNRQPVRPDQRRFAEEAEGGLCRLRILAAERGDAGFYTCKAVNEYGARQCEARLEVRAHPESRSLAVLAPLQDVDVGAGEMALFECLVAGPTDVEVDWLCRGRLLQPALLKCKMHFDGRKCKLLLTSVHEDDSGVYTCKLSTAKDELTCSARLTVRPSLAPLFTRLLEDVEVLEGRAARFDCKISGTPPPVVTWTHFGCPMEESENLRLRQDGGLHSLHIAHVGSEDEGLYAVSAVNTHGQAHCSAQLYVEEPRTAASGPSSKLEKMPSIPEEPEQGELERLSIPDFLRPLQDLEVGLAKEAMLECQVTGLPYPTISWFHNGHRIQSSDDRRMTQYRDVHRLVFPAVGPQHAGVYKSVIANKLGKAACYAHLYVTDVVPGPPDGAPQVVAVTGRMVTLTWNPPRSLDMAIDPDSLTYTVQHQVLGSDQWTALVTGLREPGWAATGLRKGVQHIFRVLSTTVKSSSKPSPPSEPVQLLEHGPTLEEAPAMLDKPDIVYVVEGQPASVTVTFNHVEAQVVWRSCRGALLEARAGVYELSQPDDDQYCLRICRVSRRDMGALTCTARNRHGTQTCSVTLELAEAPRFESIMEDVEVGAGETARFAVVVEGKPLPDIMWYKDEVLLTESSHVSFVYEENECSLVVLSTGAQDGGVYTCTAQNLAGEVSCKAELAVHSAQTAMEVEGVGEDEDHRGRRLSDFYDIHQEIGRGAFSYLRRIVERSSGLEFAAKFIPSQAKPKASARREARLLARLQHDCVLYFHEAFERRRGLVIVTELCTEELLERIARKPTVCESEIRAYMRQVLEGIHYLHQSHVLHLDVKPENLLVWDGAAGEQQVRICDFGNAQELTPGEPQYCQYGTPEFVAPEIVNQSPVSGVTDIWPVGVVAFLCLTGISPFVGENDRTTLMNIRNYNVAFEETTFLSLSREARGFLIKVLVQDRLRPTAEETLEHPWFKTQAKGAEVSTDHLKLFLSRRRWQRSQISYKCHLVLRPIPELLRAPPERVWVTMPRRPPPSGGLSSSSDSEEEELEELPSVPRPLQPEFSGSRVSLTDIPTEDEALGTPETGAATPMDWQEQGRAPSQDQEAPSPEALPSPGQEPAAGASPRRGELRRGSSAESALPRAGPRELGRGLHKAASVELPQRRSPSPGATRLARGGLGEGEYAQRLQALRQRLLRGGPEDGKVSGLRGPLLESLGGRARDPRMARAASSEAAPHHQPPLENRGLQKSSSFSQGEAEPRGRHRRAGAPLEIPVARLGARRLQESPSLSALSEAQPSSPARPSAPKPSTPKSAEPSATTPSDAPQPPAPQPAQDKAPEPRPEPVRASKPAPPPQALQTLALPLTPYAQIIQSLQLSGHAQGPSQGPAAPPSEPKPHAAVFARVASPPPGAPEKRVPSAGGPPVLAEKARVPTVPPRPGSSLSSSIENLESEAVFEAKFKRSRESPLSLGLRLLSRSRSEERGPFRGAEEEDGIYRPSPAGTPLELVRRPERSRSVQDLRAVGEPGLVRRLSLSLSQRLRRTPPAQRHPAWEARGGDGESSEGGSSARGSPVLAMRRRLSFTLERLSSRLQRSGSSEDSGGASGRSTPLFGRLRRATSEGESLRRLGLPHNQLAAQAGATTPSAESLGSEASATSGSSAPGESRSRLRWGFSRPRKDKGLSPPNLSASVQEELGHQYVRSESDFPPVFHIKLKDQVLLEGEAATLLCLPAACPAPHISWMKDKKSLRSEPSVIIVSCKDGRQLLSIPRAGKRHAGLYECSATNVLGSITSSCTVAVARVPGKLAPPEVPQTYQDTALVLWKPGDSRAPCTYTLERRVDGESVWHPVSSGIPDCYYNVTHLPVGVTVRFRVACANRAGQGPFSNSSEKVFVRGTQDSSAVPSAAHQEAPVTSRPARARPPDSPTSLAPPLAPAAPTPPSVTVSPSSPPTPPSQALSSLKAVGPPPQTPPRRHRGLQAARPAEPTLPSTHVTPSEPKPFVLDTGTPIPASTPQGVKPVSSSTPVYVVTSFVSAPPAPEPPAPEPPPEPTKVTVQSLSPAKEVVSSPGSSPRSSPRPEGTTLRQGPPQKPYTFLEEKARGRFGVVRACRENATGRTFVAKIVPYAAEGKRRVLQEYEVLRTLHHERIMSLHEAYITPRYLVLIAESCGNRELLCGLSDRFRYSEDDVATYMVQLLQGLDYLHGHHVLHLDIKPDNLLLAPDNALKIVDFGSAQPYNPQALRPLGHRTGTLEFMAPEMVKGEPIGSATDIWGAGVLTYIMLSGRSPFYEPDPQETEARIVGGRFDAFQLYPNTSQSATLFLRKVLSVHPWSRPSLQDCLAHPWLQDAYLMKLRRQTLTFTTNRLKEFLGEQRRRRAEAATRHKVLLRSYPGGP",
# KC1AL_HUMAN
"MTNNSGSKAELVVGGKYKLVRKIGSGSFGDVYLGITTTNGEDVAVKLESQKVKHPQLLYESKLYTILQGGVGIPHMHWYGQEKDNNVLVMDLLGPSLEDLFNFCSRRFTMKTVLMLADQMISRIEYVHTKNFLHRDIKPDNFLMGTGRHCNKLFLIDFGLAKKYRDNRTRQHIPYREDKHLIGTVRYASINAHLGIEQSRRDDMESLGYVFMYFNRTSLPWQGLRAMTKKQKYEKISEKKMSTPVEVLCKGFPAEFAMYLNYCRGLRFEEVPDYMYLRQLFRILFRTLNHQYDYTFDWTMLKQKAAQQAASSSGQGQQAQTQTGKQTEKNKNNVKDN",
# RIPK4_HUMAN
"MEGDGGTPWALALLRTFDAGEFTGWEKVGSGGFGQVYKVRHVHWKTWLAIKCSPSLHVDDRERMELLEEAKKMEMAKFRYILPVYGICREPVGLVMEYMETGSLEKLLASEPLPWDLRFRIIHETAVGMNFLHCMAPPLLHLDLKPANILLDAHYHVKISDFGLAKCNGLSHSHDLSMDGLFGTIAYLPPERIREKSRLFDTKHDVYSFAIVIWGVLTQKKPFADEKNILHIMVKVVKGHRPELPPVCRARPRACSHLIRLMQRCWQGDPRVRPTFQGNGLNGELIRQVLAALLPVTGRWRSPGEGFRLESEVIIRVTCPLSSPQEITSETEDLCEKPDDEVKETAHDLDVKSPPEPRSEVVPARLKRASAPTFDNDYSLSELLSQLDSGVSQAVEGPEELSRSSSESKLPSSGSGKRLSGVSSVDSAFSSRGSLSLSFEREPSTSDLGTTDVQKKKLVDAIVSGDTSKLMKILQPQDVDLALDSGASLLHLAVEAGQEECAKWLLLNNANPNLSNRRGSTPLHMAVERRVRGVVELLLARKISVNAKDEDQWTALHFAAQNGDESSTRLLLEKNASVNEVDFEGRTPMHVACQHGQENIVRILLRRGVDVSLQGKDAWLPLHYAAWQGHLPIVKLLAKQPGVSVNAQTLDGRTPLHLAAQRGHYRVARILIDLCSDVNVCSLLAQTPLHVAAETGHTSTARLLLHRGAGKEAMTSDGYTALHLAARNGHLATVKLLVEEKADVLARGPLNQTALHLAAAHGHSEVVEELVSADVIDLFDEQGLSALHLAAQGRHAQTVETLLRHGAHINLQSLKFQGGHGPAATLLRRSKT",
# NEK5_HUMAN
"MDKYDVIKAIGQGAFGKAYLAKGKSDSKHCVIKEINFEKMPIQEKEASKKEVILLEKMKHPNIVAFFNSFQENGRLFIVMEYCDGGDLMKRINRQRGVLFSEDQILGWFVQISLGLKHIHDRKILHRDIKAQNIFLSKNGMVAKLGDFGIARVLNNSMELARTCIGTPYYLSPEICQNKPYNNKTDIWSLGCVLYELCTLKHPFEGNNLQQLVLKICQAHFAPISPGFSRELHSLISQLFQVSPRDRPSINSILKRPFLENLIPKYLTPEVIQEEFSHMLICRAGAPASRHAGKVVQKCKIQKVRFQGKCPPRSRISVPIKRNAILHRNEWRPPAGAQKARSIKMIERPKIAAVCGHYDYYYAQLDMLRRRAHKPSYHPIPQENTGVEDYGQETRHGPSPSQWPAEYLQRKFEAQQYKLKVEKQLGLRPSSAEPNYNQRQELRSNGEEPRFQELPFRKNEMKEQEYWKQLEEIRQQYHNDMKEIRKKMGREPEENSKISHKTYLVKKSNLPVHQDASEGEAPVQMEFRSCCPGWSAMARSWLTATSASQDIEKDLKQMRLQNTKESKNPEQKYKAKKGVKFEINLDKCISDENILQEEEAMDIPNETLTFEDGMKFKEYECVKEHGDYTDKAFEKLHCPEAGFSTQTVAAVGNRRQWDGGAPQTLLQMMAVADITSTCPTGPDSESVLSVSRQEGKTKDPYSPVLILM",
# KCC1D_HUMAN
"MARENGESSSSWKKQAEDIKKIFEFKETLGTGAFSEVVLAEEKATGKLFAVKCIPKKALKGKESSIENEIAVLRKIKHENIVALEDIYESPNHLYLVMQLVSGGELFDRIVEKGFYTEKDASTLIRQVLDAVYYLHRMGIVHRDLKPENLLYYSQDEESKIMISDFGLSKMEGKGDVMSTACGTPGYVAPEVLAQKPYSKAVDCWSIGVIAYILLCGYPPFYDENDSKLFEQILKAEYEFDSPYWDDISDSAKDFIRNLMEKDPNKRYTCEQAARHPWIAGDTALNKNIHESVSAQIRKNFAKSKWRQAFNATAVVRHMRKLHLGSSLDSSNASVSSSLSLASQKDCLAPSTLCSFISSSSGVSGVGAERRPRPTTVTAVHSGSK",
# M3K19_HUMAN
"MSSMPKPERHAESLLDICHDTNSSPTDLMTVTKNQNIILQSISRSEEFDQDGDCSHSTLVNEEEDPSGGRQDWQPRTEGVEITVTFPRDVSPPQEMSQEDLKEKNLINSSLQEWAQAHAVSHPNEIETVELRKKKLTMRPLVLQKEESSRELCNVNLGFLLPRSCLELNISKSVTREDAPHFLKEQQRKSEEFSTSHMKYSGRSIKFLLPPLSLLPTRSGVLTIPQNHKFPKEKERNIPSLTSFVPKLSVSVRQSDELSPSNEPPGALVKSLMDPTLRSSDGFIWSRNMCSFPKTNHHRQCLEKEENWKSKEIEECNKIEITHFEKGQSLVSFENLKEGNIPAVREEDIDCHGSKTRKPEEENSQYLSSRKNESSVAKNYEQDPEIVCTIPSKFQETQHSEITPSQDEEMRNNKAASKRVSLHKNEAMEPNNILEECTVLKSLSSVVFDDPIDKLPEGCSSMETNIKISIAERAKPEMSRMVPLIHITFPVDGSPKEPVIAKPSLQTRKGTIHNNHSVNIPVHQENDKHKMNSHRSKLDSKTKTSKKTPQNFVISTEGPIKPTMHKTSIKTQIFPALGLVDPRPWQLPRFQKKMPQIAKKQSTHRTQKPKKQSFPCICKNPGTQKSCVPLSVQPTEPRLNYLDLKYSDMFKEINSTANGPGIYEMFGTPVYCHVRETERDENTYYREICSAPSGRRITNKCRSSHSERKSNIRTRLSQKKTHMKCPKTSFGIKQEHKVLISKEKSSKAVHSNLHDIENGDGISEPDWQIKSSGNEFLSSKDEIHPMNLAQTPEQSMKQNEFPPVSDLSIVEEVSMEESTGDRDISNNQILTTSLRDLQELEELHHQIPFIPSEDSWAVPSEKNSNKYVQQEKQNTASLSKVNASRILTNDLEFDSVSDHSKTLTNFSFQAKQESASSQTYQYWVHYLDHDSLANKSITYQMFGKTLSGTNSISQEIMDSVNNEELTDELLGCLAAELLALDEKDNNSCQKMANETDPENLNLVLRWRGSTPKEMGRETTKVKIQRHSSGLRIYDREEKFLISNEKKIFSENSLKSEEPILWTKGEILGKGAYGTVYCGLTSQGQLIAVKQVALDTSNKLAAEKEYRKLQEEVDLLKALKHVNIVAYLGTCLQENTVSIFMEFVPGGSISSIINRFGPLPEMVFCKYTKQILQGVAYLHENCVVHRDIKGNNVMLMPTGIIKLIDFGCARRLAWAGLNGTHSDMLKSMHGTPYWMAPEVINESGYGRKSDIWSIGCTVFEMATGKPPLASMDRMAAMFYIGAHRGLMPPLPDHFSENAADFVRMCLTRDQHERPSALQLLKHSFLERSH",
# NEK11_HUMAN
"MLKFQEAAKCVSGSTAISTYPKTLIARRYVLQQKLGSGSFGTVYLVSDKKAKRGEELKVLKEISVGELNPNETVQANLEAQLLSKLDHPAIVKFHASFVEQDNFCIITEYCEGRDLDDKIQEYKQAGKIFPENQIIEWFIQLLLGVDYMHERRILHRDLKSKNVFLKNNLLKIGDFGVSRLLMGSCDLATTLTGTPHYMSPEALKHQGYDTKSDIWSLACILYEMCCMNHAFAGSNFLSIVLKIVEGDTPSLPERYPKELNAIMESMLNKNPSLRPSAIEILKIPYLDEQLQNLMCRYSEMTLEDKNLDCQKEAAHIINAMQKRIHLQTLRALSEVQKMTPRERMRLRKLQAADEKARKLKKIVEEKYEENSKRMQELRSRNFQQLSVDVLHEKTHLKGMEEKEEQPEGRLSCSPQDEDEERWQGREEESDEPTLENLPESQPIPSMDLHELESIVEDATSDLGYHEIPEDPLVAEEYYADAFDSYCEESDEEEEEIALERPEKEIRNEGSQPAYRTNQQDSDIEALARCLENVLGCTSLDTKTITTMAEDMSPGPPIFNSVMARTKMKRMRESAMQKLGTEVFEEVYNYLKRARHQNASEAEIRECLEKVVPQASDCFEVDQLLYFEEQLLITMGKEPTLQNHL",
# GRK7_HUMAN
"MVDMGALDNLIANTAYLQARKPSDCDSKELQRRRRSLALPGLQGCAELRQKLSLNFHSLCEQQPIGRRLFRDFLATVPTFRKAATFLEDVQNWELAEEGPTKDSALQGLVATCASAPAPGNPQPFLSQAVATKCQAATTEEERVAAVTLAKAEAMAFLQEQPFKDFVTSAFYDKFLQWKLFEMQPVSDKYFTEFRVLGKGGFGEVCAVQVKNTGKMYACKKLDKKRLKKKGGEKMALLEKEILEKVSSPFIVSLAYAFESKTHLCLVMSLMNGGDLKFHIYNVGTRGLDMSRVIFYSAQIACGMLHLHELGIVYRDMKPENVLLDDLGNCRLSDLGLAVEMKGGKPITQRAGTNGYMAPEILMEKVSYSYPVDWFAMGCSIYEMVAGRTPFKDYKEKVSKEDLKQRTLQDEVKFQHDNFTEEAKDICRLFLAKKPEQRLGSREKSDDPRKHHFFKTINFPRLEAGLIEPPFVPDPSVVYAKDIAEIDDFSEVRGVEFDDKDKQFFKNFATGAVPIAWQEEIIETGLFEELNDPNRPTGCEEGNSSKSGVCLLL",
# SBK2_HUMAN
"MPGKQSEEGPAEAGASEDSEEEGLGGLTLEELQQGQEAARALEDMMTLSAQTLVRAEVDELYEEVRPLGQGCYGRVLLVTHRQKGTPLALKQLPKPRTSLRGFLYEFCVGLSLGAHSAIVTAYGIGIESAHSYSFLTEPVLHGDLMAFIQPKVGLPQPAVHRCAAQLASALEYIHARGLVYRDLKPENVLVCDPACRRFKLTDFGHTRPRGTLLRLAGPPIPYTAPELCAPPPLPEGLPIQPALDAWALGVLLFCLLTGYFPWDRPLAEADPFYEDFLIWQASGQPRDRPQPWFGLAAAADALLRGLLDPHPRRRSAVIAIREHLGRPWRQREGEAEAVGAVEEEAGQ",
# HIPK4_HUMAN
"MSTIQSETDCYDIIEVLGKGTFGEVAKGWRRSTGEMVAIKILKNDAYRNRIIKNELKLLHCMRGLDPEEAHVIRFLEFFHDALKFYLVFELLEQNLFEFQKENNFAPLPARHIRTVTLQVLTALARLKELAIIHADLKPENIMLVDQTRCPFRVKVIDFGSASIFSEVRYVKEPYIQSRFYRAPEILLGLPFCEKVDVWSLGCVMAELHLGWPLYPGNNEYDQVRYICETQGLPKPHLLHAACKAHHFFKRNPHPDAANPWQLKSSADYLAETKVRPLERRKYMLKSLDQIETVNGGSVASRLTFPDREALAEHADLKSMVELIKRMLTWESHERISPSAALRHPFVSMQQLRSAHETTHYYQLSLRSYRLSLQVEGKPPTPVVAAEDGTPYYCLAEEKEAAGMGSVAGSSPFFREEKAPGMQRAIDQLDDLSLQEAGHGLWGETCTNAVSDMMVPLKAAITGHHVPDSGPEPILAFYSSRLAGRHKARKPPAGSKSDSNFSNLIRLSQVSPEDDRPCRGSSWEEGEHLGASAEPLAILQRDEDGPNIDNMTMEAERPDPELFDPSSCPGEWLSEPDCTLESVRGPRAQGLPPRRSHQHGPPRGATSFLQHVTGHH",
# MYO3B_HUMAN
"MKHLYGLFHYNPMMLGLESLPDPTDTWEIIETIGKGTYGKVYKVTNKRDGSLAAVKILDPVSDMDEEIEAEYNILQFLPNHPNVVKFYGMFYKADHCVGGQLWLVLELCNGGSVTELVKGLLRCGQRLDEAMISYILYGALLGLQHLHNNRIIHRDVKGNNILLTTEGGVKLVDFGVSAQLTSTRLRRNTSVGTPFWMAPEVIACEQQYDSSYDARCDVWSLGITAIELGDGDPPLFDMHPVKTLFKIPRNPPPTLLHPEKWCEEFNHFISQCLIKDFERRPSVTHLLDHPFIKGVHGKVLFLQKQLAKVLQDQKHQNPVAKTRHERMHTRRPYHVEDAEKYCLEDDLVNLEVLDEDTIIHQLQKRYADLLIYTYVGDILIALNPFQNLSIYSPQFSRLYHGVKRASNPPHIFASADAAYQCMVTLSKDQCIVISGESGSGKTESAHLIVQHLTFLGKANNQTLREKILQVNSLVEAFGNSCTAINDNSSRFGKYLEMMFTPTGVVMGARISEYLLEKSRVIKQAAREKNFHIFYYIYAGLHHQKKLSDFRLPEEKPPRYIADETGRVMHDITSKESYRRQFEAIQHCFRIIGFTDKEVHSVYRILAGILNIGNIEFAAISSQHQTDKSEVPNAEALQNAASVLCISPEELQEALTSHCVVTRGETIIRANTVDRAADVRDAMSKALYGRLFSWIVNRINTLLQPDENICSAGGGMNVGILDIFGFENFQRNSFEQLCINIANEQIQYYFNQHVFALEQMEYQNEGIDAVPVEYEDNRPLLDMFLQKPLGLLALLDEESRFPQATDQTLVDKFEDNLRCKYFWRPKGVELCFGIQHYAGKVLYDASGVLEKNRDTLPADVVVVLRTSENKLLQQLFSIPLTKTGNLAQTRARITVASSSLPPHFSAGKAKVDTLEVIRHPEETTNMKRQTVASYFRYSLMDLLSKMVVGQPHFVRCIKPNDDREALQFSRERVLAQLRSTGILETVSIRRQGYSHRILFEEFVKRYYYLAFTAHQTPLASKESCVAILEKSRLDHWVLGKTKVFLKYYHVEQLNLLLREVIGRVVVLQAYTKGWLGARRYKRVREKREKGAIAIQSAWRGYDARRKFKKISNRRNESAAHNQAGDTSNQSSGPHSPVAAGTRGSAEVQDCSEPGDHKVLRGSVHRRSHSQAESNNGRTQTSSNSPAVTEKNGHSQAQSSPKGCDIFAGHANKHSVSGTDLLSSRICHPAPDQQGLSLWGAPQKPGSENGLAQKHRTPRRRCQQPKMLSSPEDTMYYNQLNGTLEYQGSKRKPRKLGQIKVLDGEDEYYKSLSPVDCIPEENNSAHPSFFSSSSKGDSFAQH",
# WNK4_HUMAN
"MLASPATETTVLMSQTEADLALRPPPPLGTAGQPRLGPPPRRARRFSGKAEPRPRSSRLSRRSSVDLGLLSSWSLPASPAPDPPDPPDSAGPGPARSPPPSSKEPPEGTWTEGAPVKAAEDSARPELPDSAVGPGSREPLRVPEAVALERRREQEEKEDMETQAVATSPDGRYLKFDIEIGRGSFKTVYRGLDTDTTVEVAWCELQTRKLSRAERQRFSEEVEMLKGLQHPNIVRFYDSWKSVLRGQVCIVLVTELMTSGTLKTYLRRFREMKPRVLQRWSRQILRGLHFLHSRVPPILHRDLKCDNVFITGPTGSVKIGDLGLATLKRASFAKSVIGTPEFMAPEMYEEKYDEAVDVYAFGMCMLEMATSEYPYSECQNAAQIYRKVTSGRKPNSFHKVKIPEVKEIIEGCIRTDKNERFTIQDLLAHAFFREERGVHVELAEEDDGEKPGLKLWLRMEDARRGGRPRDNQAIEFLFQLGRDAAEEVAQEMVALGLVCEADYQPVARAVRERVAAIQRKREKLRKARELEALPPEPGPPPATVPMAPGPPSVFPPEPEEPEADQHQPFLFRHASYSSTTSDCETDGYLSSSGFLDASDPALQPPGGVPSSLAESHLCLPSAFALSIPRSGPGSDFSPGDSYASDAASGLSDVGEGMGQMRRPPGRNLRRRPRSRLRVTSVSDQNDRVVECQLQTHNSKMVTFRFDLDGDSPEEIAAAMVYNEFILPSERDGFLRRIREIIQRVETLLKRDTGPMEAAEDTLSPQEEPAPLPALPVPLPDPSNEELQSSTSLEHRSWTAFSTSSSSPGTPLSPGNPFSPGTPISPGPIFPITSPPCHPSPSPFSPISSQVSSNPSPHPTSSPLPFSSSTPEFPVPLSQCPWSSLPTTSPPTFSPTCSQVTLSSPFFPPCPSTSSFPSTTAAPLLSLASAFSLAVMTVAQSLLSPSPGLLSQSPPAPPSPLPSLPLPPPVAPGGQESPSPHTAEVESEASPPPARPLPGEARLAPISEEGKPQLVGRFQVTSSKEPAEPLPLQPTSPTLSGSPKPSTPQLTSESSDTEDSAGGGPETREALAESDRAAEGLGAGVEEEGDDGKEPQVGGSPQPLSHPSPVWMNYSYSSLCLSSEESESSGEDEEFWAELQSLRQKHLSEVETLQTLQKKEIEDLYSRLGKQPPPGIVAPAAMLSSRQRRLSKGSFPTSRRNSLQRSEPPGPGIMRRNSLSGSSTGSQEQRASKGVTFAGDVGRM",
# SBK3_HUMAN
"MERRASETPEDGDPEEDTATALQRLVELTTSRVTPVRSLRDQYHLIRKLGSGSYGRVLLAQPHQGGPAVALKLLRRDLVLRSTFLREFCVGRCVSAHPGLLQTLAGPLQTPRYFAFAQEYAPCGDLSGMLQERGLPELLVKRVVAQLAGALDFLHSRGLVHADVKPDNVLVFDPVCSRVALGDLGLTRPEGSPTPAPPVPLPTAPPELCLLLPPDTLPLRPAVDSWGLGVLLFCAATACFPWDVALAPNPEFEAFAGWVTTKPQPPQPPPPWDQFAPPALALLQGLLDLDPETRSPPLAVLDFLGDDWGLQGNREGPGVLGSAVSYEDREEGGSSLEEWTDEGDDSKSGGRTGTDGGAP",
# BRSK1_HUMAN
"MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTGLVKLGVHCITGQKVAIKIVNREKLSESVLMKVEREIAILKLIEHPHVLKLHDVYENKKYLYLVLEHVSGGELFDYLVKKGRLTPKEARKFFRQIVSALDFCHSYSICHRDLKPENLLLDEKNNIRIADFGMASLQVGDSLLETSCGSPHYACPEVIKGEKYDGRRADMWSCGVILFALLVGALPFDDDNLRQLLEKVKRGVFHMPHFIPPDCQSLLRGMIEVEPEKRLSLEQIQKHPWYLGGKHEPDPCLEPAPGRRVAMRSLPSNGELDPDVLESMASLGCFRDRERLHRELRSEEENQEKMIYYLLLDRKERYPSCEDQDLPPRNDVDPPRKRVDSPMLSRHGKRRPERKSMEVLSITDAGGGGSPVPTRRALEMAQHSQRSRSVSGASTGLSSSPLSSPRSPVFSFSPEPGAGDEARGGGSPTSKTQTLPSRGPRGGGAGEQPPPPSARSTPLPGPPGSPRSSGGTPLHSPLHTPRASPTGTPGTTPPPSPGGGVGGAAWRSRLNSIRNSFLGSPRFHRRKMQVPTAEEMSSLTPESSPELAKRSWFGNFISLDKEEQIFLVLKDKPLSSIKADIVHAFLSIPSLSHSVLSQTSFRAEYKASGGPSVFQKPVRFQVDISSSEGPEPSPRRDGSGGGGIYSVTFTLISGPSRRFKRVVETIQAQLLSTHDQPSVQALADEKNGAQTRPAGAPPRSLQPPPGRPDPELSSSPRRGPPKDKKLLATNGTPLP",
# OBSCN_HUMAN
"MDQPQFSGAPRFLTRPKAFVVSVGKDATLSCQIVGNPTPQVSWEKDQQPVAAGARFRLAQDGDLYRLTILDLALGDSGQYVCRARNAIGEAFAAVGLQVDAEAACAEQAPHFLLRPTSIRVREGSEATFRCRVGGSPRPAVSWSKDGRRLGEPDGPRVRVEELGEASALRIRAARPRDGGTYEVRAENPLGAASAAAALVVDSDAADTASRPGTSTAALLAHLQRRREAMRAEGAPASPPSTGTRTCTVTEGKHARLSCYVTGEPKPETVWKKDGQLVTEGRRHVVYEDAQENFVLKILFCKQSDRGLYTCTASNLVGQTYSSVLVVVREPAVPFKKRLQDLEVREKESATFLCEVPQPSTEAAWFKEETRLWASAKYGIEEEGTERRLTVRNVSADDDAVYICETPEGSRTVAELAVQGNLLRKLPRKTAVRVGDTAMFCVELAVPVGPVHWLRNQEEVVAGGRVAISAEGTRHTLTISQCCLEDVGQVAFMAGDCQTSTQFCVSAPRKPPLQPPVDPVVKARMESSVILSWSPPPHGERPVTIDGYLVEKKKLGTYTWIRCHEAEWVATPELTVADVAEEGNFQFRVSALNSFGQSPYLEFPGTVHLAPKLAVRTPLKAVQAVEGGEVTFSVDLTVASAGEWFLDGQALKASSVYEIHCDRTRHTLTIREVPASLHGAQLKFVANGIESSIRMEVRAAPGLTANKPPAAAAREVLARLHEEAQLLAELSDQAAAVTWLKDGRTLSPGPKYEVQASAGRRVLLVRDVARDDAGLYECVSRGGRIAYQLSVQGLARFLHKDMAGSCVDAVAGGPAQFECETSEAHVHVHWYKDGMELGHSGERFLQEDVGTRHRLVAATVTRQDEGTYSCRVGEDSVDFRLRVSEPKVVFAKEQLARRKLQAEAGASATLSCEVAQAQTEVTWYKDGKKLSSSSKVCMEATGCTRRLVVQQAGQADAGEYSCEAGGQRLSFHLDVKEPKVVFAKDQVAHSEVQAEAGASATLSCEVAQAQTEVMWYKDGKKLSSSLKVHVEAKGCRRRLVVQQAGKTDAGDYSCEARGQRVSFRLHITEPKMMFAKEQSVHNEVQAEAGASAMLSCEVAQAQTEVTWYKDGKKLSSSSKVGMEVKGCTRRLVLPQAGKADAGEYSCEAGGQRVSFHLHITEPKGVFAKEQSVHNEVQAEAGTTAMLSCEVAQPQTEVTWYKDGKKLSSSSKVRMEVKGCTRRLVVQQVGKADAGEYSCEAGGQRVSFQLHITEPKAVFAKEQLVHNEVRTEAGASATLSCEVAQAQTEVTWYKDGKKLSSSSKVRIEAAGCMRQLVVQQAGQADAGEYTCEAGGQRLSFHLDVSEPKAVFAKEQLAHRKVQAEAGAIATLSCEVAQAQTEVTWYKDGKKLSSSSKVRMEAVGCTRRLVVQQACQADTGEYSCEAGGQRLSFSLDVAEPKVVFAKEQPVHREVQAQAGASTTLSCEVAQAQTEVMWYKDGKKLSFSSKVRMEAVGCTRRLVVQQAGQAVAGEYSCEAGSQRLSFHLHVAEPKAVFAKEQPASREVQAEAGTSATLSCEVAQAQTEVTWYKDGKKLSSSSKVRMEAVGCTRRLVVQEAGQADAGEYSCKAGDQRLSFHLHVAEPKVVFAKEQPAHREVQAEAGASATLSCEVAQAQTEVTWYKDGKKLSSSSKVRVEAVGCTRRLVVQQAGQAEAGEYSCEAGGQQLSFRLHVAELEPQISERPCRREPLVVKEHEDIILTATLATPSAATVTWLKDGVEIRRSKRHETASQGDTHTLTVHGAQVLDSAIYSCRVGAEGQDFPVQVEEVAAKFCRLLEPVCGELGGTVTLACELSPACAEVVWRCGNTQLRVGKRFQMVAEGPVRSLTVLGLRAEDAGEYVCESRDDHTSAQLTVSVPRVVKFMSGLSTVVAEEGGEATFQCVVSPSDVAVVWFRDGALLQPSEKFAISQSGASHSLTISDLVLEDAGQITVEAEGASSSAALRVREAPVLFKKKLEPQTVEERSSVTLEVELTRPWPELRWTRNATALAPGKNVEIHAEGARHRLVLHNVGFADRGFFGCETPDDKTQAKLTVEMRQVRLVRGLQAVEAREQGTATMEVQLSHADVDGSWTRDGLRFQQGPTCHLAVRGPMHTLTLSGLRPEDSGLMVFKAEGVHTSARLVVTELPVSFSRPLQDVVTTEKEKVTLECELSRPNVDVRWLKDGVELRAGKTMAIAAQGACRSLTIYRCEFADQGVYVCDAHDAQSSASVKVQGRTYTLIYRRVLAEDAGEIQFVAENAESRAQLRVKELPVTLVRPLRDKIAMEKHRGVLECQVSRASAQVRWFKGSQELQPGPKYELVSDGLYRKLIISDVHAEDEDTYTCDAGDVKTSAQFFVEEQSITIVRGLQDVTVMEPAPAWFECETSIPSVRPPKWLLGKTVLQAGGNVGLEQEGTVHRLMLRRTCSTMTGPVHFTVGKSRSSARLVVSDIPVVLTRPLEPKTGRELQSVVLSCDFRPAPKAVQWYKDDTPLSPSEKFKMSLEGQMAELRILRLMPADAGVYRCQAGSAHSSTEVTVEAREVTVTGPLQDAEATEEGWASFSCELSHEDEEVEWSLNGMPLYNDSFHEISHKGRRHTLVLKSIQRADAGIVRASSLKVSTSARLEVRVKPVVFLKALDDLSAEERGTLALQCEVSDPEAHVVWRKDGVQLGPSDKYDFLHTAGTRGLVVHDVSPEDAGLYTCHVGSEETRARVRVHDLHVGITKRLKTMEVLEGESCSFECVLSHESASDPAMWTVGGKTVGSSSRFQATRQGRKYILVVREAAPSDAGEVVFSVRGLTSKASLIVRERPAAIIKPLEDQWVAPGEDVELRCELSRAGTPVHWLKDRKAIRKSQKYDVVCEGTMAMLVIRGASLKDAGEYTCEVEASKSTASLHVEEKANCFTEELTNLQVEEKGTAVFTCKTEHPAATVTWRKGLLELRASGKHQPSQEGLTLRLTISALEKADSDTYTCDIGQAQSRAQLLVQGRRVHIIEDLEDVDVQEGSSATFRCRISPANYEPVHWFLDKTPLHANELNEIDAQPGGYHVLTLRQLALKDSGTIYFEAGDQRASAALRVTEKPSVFSRELTDATITEGEDLTLVCETSTCDIPVCWTKDGKTLRGSARCQLSHEGHRAQLLITGATLQDSGRYKCEAGGACSSSIVRVHARPVRFQEALKDLEVLEGGAATLRCVLSSVAAPVKWCYGNNVLRPGDKYSLRQEGAMLELVVRNLRPQDSGRYSCSFGDQTTSATLTVTALPAQFIGKLRNKEATEGATATLRCELSKAAPVEWRKGSETLRDGDRYCLRQDGAMCELQIRGLAMVDAAEYSCVCGEERTSASLTIRPMPAHFIGRLRHQESIEGATATLRCELSKAAPVEWRKGRESLRDGDRHSLRQDGAVCELQICGLAVADAGEYSCVCGEERTSATLTVKALPAKFTEGLRNEEAVEGATAMLWCELSKVAPVEWRKGPENLRDGDRYILRQEGTRCELQICGLAMADAGEYLCVCGQERTSATLTIRALPARFIEDVKNQEAREGATAVLQCELNSAAPVEWRKGSETLRDGDRYSLRQDGTKCELQIRGLAMADTGEYSCVCGQERTSAMLTVRALPIKFTEGLRNEEATEGATAVLRCELSKMAPVEWWKGHETLRDGDRHSLRQDGARCELQIRGLVAEDAGEYLCMCGKERTSAMLTVRAMPSKFIEGLRNEEATEGDTATLWCELSKAAPVEWRKGHETLRDGDRHSLRQDGSRCELQIRGLAVVDAGEYSCVCGQERTSATLTVRALPARFIEDVKNQEAREGATAVLQCELSKAAPVEWRKGSETLRGGDRYSLRQDGTRCELQIHGLSVADTGEYSCVCGQERTSATLTVKAPQPVFREPLQSLQAEEGSTATLQCELSEPTATVVWSKGGLQLQANGRREPRLQGCTAELVLQDLQREDTGEYTCTCGSQATSATLTVTAAPVRFLRELQHQEVDEGGTAHLCCELSRAGASVEWRKGSLQLFPCAKYQMVQDGAAAELLVRGVEQEDAGDYTCDTGHTQSMASLSVRVPRPKFKTRLQSLEQETGDIARLCCQLSDAESGAVVQWLKEGVELHAGPKYEMRSQGATRELLIHQLEAKDTGEYACVTGGQKTAASLRVTEPEVTIVRGLVDAEVTADEDVEFSCEVSRAGATGVQWCLQGLPLQSNEVTEVAVRDGRIHTLRLKGVTPEDAGTVSFHLGNHASSAQLTVRAPEVTILEPLQDVQLSEGQDASFQCRLSRASGQEARWALGGVPLQANEMNDITVEQGTLHLLTLHKVTLEDAGTVSFHVGTCSSEAQLKVTAKNTVVRGLENVEALEGGEALFECQLSQPEVAAHTWLLDDEPVHTSENAEVVFFENGLRHLLLLKNLRPQDSCRVTFLAGDMVTSAFLTVRGWRLEILEPLKNAAVRAGAQACFTCTLSEAVPVGEASWYINGAAVQPDDSDWTVTADGSHHALLLRSAQPHHAGEVTFACRDAVASARLTVLGLPDPPEDAEVVARSSHTVTLSWAAPMSDGGGGLCGYRVEVKEGATGQWRLCHELVPGPECVVDGLAPGETYRFRVAAVGPVGAGEPVHLPQTVRLAEPPKPVPPQPSAPESRQVAAGEDVSLELEVVAEAGEVIWHKGMERIQPGGRFEVVSQGRQQMLVIKGFTAEDQGEYHCGLAQGSICPAAATFQVALSPASVDEAPQPSLPPEAAQEGDLHLLWEALARKRRMSREPTLDSISELPEEDGRSQRLPQEAEEVAPDLSEGYSTADELARTGDADLSHTSSDDESRAGTPSLVTYLKKAGRPGTSPLASKVGAPAAPSVKPQQQQEPLAAVRPPLGDLSTKDLGDPSMDKAAVKIQAAFKGYKVRKEMKQQEGPMFSHTFGDTEAQVGDALRLECVVASKADVRARWLKDGVELTDGRHHHIDQLGDGTCSLLITGLDRADAGCYTCQVSNKFGQVTHSACVVVSGSESEAESSSGGELDDAFRRAARRLHRLFRTKSPAEVSDEELFLSADEGPAEPEEPADWQTYREDEHFICIRFEALTEARQAVTRFQEMFATLGIGVEIKLVEQGPRRVEMCISKETPAPVVPPEPLPSLLTSDAAPVFLTELQNQEVQDGYPVSFDCVVTGQPMPSVRWFKDGKLLEEDDHYMINEDQQGGHQLIITAVVPADMGVYRCLAENSMGVSSTKAELRVDLTSTDYDTAADATESSSYFSAQGYLSSREQEGTESTTDEGQLPQVVEELRDLQVAPGTRLAKFQLKVKGYPAPRLYWFKDGQPLTASAHIRMTDKKILHTLEIISVTREDSGQYAAYISNAMGAAYSSARLLVRGPDEPEEKPASDVHEQLVPPRMLERFTPKKVKKGSSITFSVKVEGRPVPTVHWLREEAERGVLWIGPDTPGYTVASSAQQHSLVLLDVGRQHQGTYTCIASNAAGQALCSASLHVSGLPKVEEQEKVKEALISTFLQGTTQAISAQGLETASFADLGGQRKEEPLAAKEALGHLSLAEVGTEEFLQKLTSQITEMVSAKITQAKLQVPGGDSDEDSKTPSASPRHGRSRPSSSIQESSSESEDGDARGEIFDIYVVTADYLPLGAEQDAITLREGQYVEVLDAAHPLRWLVRTKPTKSSPSRQGWVSPAYLDRRLKLSPEWGAAEAPEFPGEAVSEDEYKARLSSVIQELLSSEQAFVEELQFLQSHHLQHLERCPHVPIAVAGQKAVIFRNVRDIGRFHSSFLQELQQCDTDDDVAMCFIKNQAAFEQYLEFLVGRVQAESVVVSTAIQEFYKKYAEEALLAGDPSQPPPPPLQHYLEQPVERVQRYQALLKELIRNKARNRQNCALLEQAYAVVSALPQRAENKLHVSLMENYPGTLQALGEPIRQGHFIVWEGAPGARMPWKGHNRHVFLFRNHLVICKPRRDSRTDTVSYVFRNMMKLSSIDLNDQVEGDDRAFEVWQEREDSVRKYLLQARTAIIKSSWVKEICGIQQRLALPVWRPPDFEEELADCTAELGETVKLACRVTGTPKPVISWYKDGKAVQVDPHHILIEDPDGSCALILDSLTGVDSGQYMCFAASAAGNCSTLGKILVQVPPRFVNKVRASPFVEGEDAQFTCTIEGAPYPQIRWYKDGALLTTGNKFQTLSEPRSGLLVLVIRAASKEDLGLYECELVNRLGSARASAELRIQSPMLQAQEQCHREQLVAAVEDTTLERADQEVTSVLKRLLGPKAPGPSTGDLTGPGPCPRGAPALQETGSQPPVTGTSEAPAVPPRVPQPLLHEGPEQEPEAIARAQEWTVPIRMEGAAWPGAGTGELLWDVHSHVVRETTQRTYTYQAIDTHTARPPSMQVTIEDVQAQTGGTAQFEAIIEGDPQPSVTWYKDSVQLVDSTRLSQQQEGTTYSLVLRHVASKDAGVYTCLAQNTGGQVLCKAELLVLGGDNEPDSEKQSHRRKLHSFYEVKEEIGRGVFGFVKRVQHKGNKILCAAKFIPLRSRTRAQAYRERDILAALSHPLVTGLLDQFETRKTLILILELCSSEELLDRLYRKGVVTEAEVKVYIQQLVEGLHYLHSHGVLHLDIKPSNILMVHPAREDIKICDFGFAQNITPAELQFSQYGSPEFVSPEIIQQNPVSEASDIWAMGVISYLSLTCSSPFAGESDRATLLNVLEGRVSWSSPMAAHLSEDAKDFIKATLQRAPQARPSAAQCLSHPWFLKSMPAEEAHFINTKQLKFLLARSRWQRSLMSYKSILVMRSIPELLRGPPDSPSLGVARHLCRDTGGSSSSSSSSDNELAPFARAKSLPPSPVTHSPLLHPRGFLRPSASLPEEAEASERSTEAPAPPASPEGAGPPAAQGCVPRHSVIRSLFYHQAGESPEHGALAPGSRRHPARRRHLLKGGYIAGALPGLREPLMEHRVLEEEAAREEQATLLAKAPSFETALRLPASGTHLAPGHSHSLEHDSPSTPRPSSEACGEAQRLPSAPSGGAPIRDMGHPQGSKQLPSTGGHPGTAQPERPSPDSPWGQPAPFCHPKQGSAPQEGCSPHPAVAPCPPGSFPPGSCKEAPLVPSSPFLGQPQAPPAPAKASPPLDSKMGPGDISLPGRPKPGPCSSPGSASQASSSQVSSLRVGSSQVGTEPGPSLDAEGWTQEAEDLSDSTPTLQRPQEQATMRKFSLGGRGGYAGVAGYGTFAFGGDAGGMLGQGPMWARIAWAVSQSEEEEQEEARAESQSEEQQEARAESPLPQVSARPVPEVGRAPTRSSPEPTPWEDIGQVSLVQIRDLSGDAEAADTISLDISEVDPAYLNLSDLYDIKYLPFEFMIFRKVPKSAQPEPPSPMAEEELAEFPEPTWPWPGELGPHAGLEITEESEDVDALLAEAAVGRKRKWSSPSRSLFHFPGRHLPLDEPAELGLRERVKASVEHISRILKGRPEGLEKEGPPRKKPGLASFRLSGLKSWDRAPTFLRELSDETVVLGQSVTLACQVSAQPAAQATWSKDGAPLESSSRVLISATLKNFQLLTILVVVAEDLGVYTCSVSNALGTVTTTGVLRKAERPSSSPCPDIGEVYADGVLLVWKPVESYGPVTYIVQCSLEGGSWTTLASDIFDCCYLTSKLSRGGTYTFRTACVSKAGMGPYSSPSEQVLLGGPSHLASEEESQGRSAQPLPSTKTFAFQTQIQRGRFSVVRQCWEKASGRALAAKIIPYHPKDKTAVLREYEALKGLRHPHLAQLHAAYLSPRHLVLILELCSGPELLPCLAERASYSESEVKDYLWQMLSATQYLHNQHILHLDLRSENMIITEYNLLKVVDLGNAQSLSQEKVLPSDKFKDYLETMAPELLEGQGAVPQTDIWAIGVTAFIMLSAEYPVSSEGARDLQRGLRKGLVRLSRCYAGLSGGAVAFLRSTLCAQPWGRPCASSCLQCPWLTEEGPACSRPAPVTFPTARLRVFVRNREKRRALLYKRHNLAQVR",
# PSKH2_HUMAN
"MGCGASRKVVPGPPALAWAKHEGQNQAGVGGAGPGPEAAAQAAQRIQVARFRAKFDPRVLARYDIKALIGTGSFSRVVRVEQKTTKKPFAIKVMETREREGREACVSELSVLRRVSHRYIVQLMEIFETEDQVYMVMELATGGELFDRLIAQGSFTERDAVRILQMVADGIRYLHALQITHRNLKPENLLYYHPGEESKILITDFGLAYSGKKSGDWTMKTLCGTPEYIAPEVLLRKPYTSAVDMWALGVITYALLSGFLPFDDESQTRLYRKILKGKYNYTGEPWPSISHLAKDFIDKLLILEAGHRMSAGQALDHPWVITMAAGSSMKNLQRAISRNLMQRASPHSQSPGSAQSSKSHYSHKSRHMWSKRNLRIVESPLSALL",
# SIK1_HUMAN
"MVIMSEFSADPAGQGQGQQKPLRVGFYDIERTLGKGNFAVVKLARHRVTKTQVAIKIIDKTRLDSSNLEKIYREVQLMKLLNHPHIIKLYQVMETKDMLYIVTEFAKNGEMFDYLTSNGHLSENEARKKFWQILSAVEYCHDHHIVHRDLKTENLLLDGNMDIKLADFGFGNFYKSGEPLSTWCGSPPYAAPEVFEGKEYEGPQLDIWSLGVVLYVLVCGSLPFDGPNLPTLRQRVLEGRFRIPFFMSQDCESLIRRMLVVDPARRITIAQIRQHRWMRAEPCLPGPACPAFSAHSYTSNLGDYDEQALGIMQTLGVDRQRTVESLQNSSYNHFAAIYYLLLERLKEYRNAQCARPGPARQPRPRSSDLSGLEVPQEGLSTDPFRPALLCPQPQTLVQSVLQAEMDCELQSSLQWPLFFPVDASCSGVFRPRPVSPSSLLDTAISEEARQGPGLEEEQDTQESLPSSTGRRHTLAEVSTRLSPLTAPCIVVSPSTTASPAEGTSSDSCLTFSASKSPAGLSGTPATQGLLGACSPVRLASPFLGSQSATPVLQAQGGLGGAVLLPVSFQEGRRASDTSLTQGLKAFRQQLRKTTRTKGFLGLNKIKGLARQVCQAPASRASRGGLSPFHAPAQSPGLHGGAAGSREGWSLLEEVLEQQRLLQLQHHPAAAPGCSQAPQPAPAPFVIAPCDGPGAAPLPSTLLTSGLPLLPPPLLQTGASPVASAAQLLDTHLHIGTGPTALPAVPPPRLARLAPGCEPLGLLQGDCEMEDLMPCSLGTFVLVQ",
# KSR2_HUMAN
"MDEENMTKSEEQQPLSLQKALQQCELVQNMIDLSISNLEGLRTKCATSNDLTQKEIRTLESKLVKYFSRQLSCKKKVALQERNAELDGFPQLRHWFRIVDVRKEVLEEISPGQLSLEDLLEMTDEQVCETVEKYGANREECARLNASLSCLRNVHMSGGNLSKQDWTIQWPTTETGKENNPVCPPEPTPWIRTHLSQSPRVPSKCVQHYCHTSPTPGAPVYTHVDRLTVDAYPGLCPPPPLESGHRSLPPSPRQRHAVRTPPRTPNIVTTVTPPGTPPMRKKNKLKPPGTPPPSSRKLIHLIPGFTALHRSKSHEFQLGHRVDEAHTPKAKKKSKPLNLKIHSSVGSCENIPSQQRSPLLSERSLRSFFVGHAPFLPSTPPVHTEANFSANTLSVPRWSPQIPRRDLGNSIKHRFSTKYWMSQTCTVCGKGMLFGLKCKNCKLKCHNKCTKEAPPCHLLIIHRGDPARLVRTESVPCDINNPLRKPPRYSDLHISQTLPKTNKINKDHIPVPYQPDSSSNPSSTTSSTPSSPAPPLPPSATPPSPLHPSPQCTRQQKNFNLPASHYYKYKQQFIFPDVVPVPETPTRAPQVILHPVTSNPILEGNPLLQIEVEPTSENEEVHDEAEESEDDFEEMNLSLLSARSFPRKASQTSIFLQEWDIPFEQLEIGELIGKGRFGQVYHGRWHGEVAIRLIDIERDNEDQLKAFKREVMAYRQTRHENVVLFMGACMSPPHLAIITSLCKGRTLYSVVRDAKIVLDVNKTRQIAQEIVKGMGYLHAKGILHKDLKSKNVFYDNGKVVITDFGLFSISGVLQAGRREDKLRIQNGWLCHLAPEIIRQLSPDTEEDKLPFSKHSDVFALGTIWYELHAREWPFKTQPAEAIIWQMGTGMKPNLSQIGMGKEISDILLFCWAFEQEERPTFTKLMDMLEKLPKRNRRLSHPGHFWKSAEL",
# RIOK3_HUMAN
"MDLVGVASPEPGTAAAWGPSKCPWAIPQNTISCSLADVMSEQLAKELQLEEEAAVFPEVAVAEGPFITGENIDTSSDLMLAQMLQMEYDREYDAQLRREEKKFNGDSKVSISFENYRKVHPYEDSDSSEDEVDWQDTRDDPYRPAKPVPTPKKGFIGKGKDITTKHDEVVCGRKNTARMENFAPEFQVGDGIGMDLKLSNHVFNALKQHAYSEERRSARLHEKKEHSTAEKAVDPKTRLLMYKMVNSGMLETITGCISTGKESVVFHAYGGSMEDEKEDSKVIPTECAIKVFKTTLNEFKNRDKYIKDDFRFKDRFSKLNPRKIIRMWAEKEMHNLARMQRAGIPCPTVVLLKKHILVMSFIGHDQVPAPKLKEVKLNSEEMKEAYYQTLHLMRQLYHECTLVHADLSEYNMLWHAGKVWLIDVSQSVEPTHPHGLEFLFRDCRNVSQFFQKGGVKEALSERELFNAVSGLNITADNEADFLAEIEALEKMNEDHVQKNGRKAASFLKDDGDPPLLYDE",
# COQ8A_HUMAN
"MAAILGDTIMVAKGLVKLTQAAVETHLQHLGIGGELIMAARALQSTAVEQIGMFLGKVQGQDKHEEYFAENFGGPEGEFHFSVPHAAGASTDFSSASAPDQSAPPSLGHAHSEGPAPAYVASGPFREAGFPGQASSPLGRANGRLFANPRDSFSAMGFQRRFFHQDQSPVGGLTAEDIEKARQAKARPENKQHKQTLSEHARERKVPVTRIGRLANFGGLAVGLGFGALAEVAKKSLRSEDPSGKKAVLGSSPFLSEANAERIVRTLCKVRGAALKLGQMLSIQDDAFINPHLAKIFERVRQSADFMPLKQMMKTLNNDLGPNWRDKLEYFEERPFAAASIGQVHLARMKGGREVAMKIQYPGVAQSINSDVNNLMAVLNMSNMLPEGLFPEHLIDVLRRELALECDYQREAACARKFRDLLKGHPFFYVPEIVDELCSPHVLTTELVSGFPLDQAEGLSQEIRNEICYNILVLCLRELFEFHFMQTDPNWSNFFYDPQQHKVALLDFGATREYDRSFTDLYIQIIRAAADRDRETVRAKSIEMKFLTGYEVKVMEDAHLDAILILGEAFASDEPFDFGTQSTTEKIHNLIPVMLRHRLVPPPEETYSLHRKMGGSFLICSKLKARFPCKAMFEEAYSNYCKRQAQQ",
# RIOK1_HUMAN
"MDYRRLLMSRVVPGQFDDADSSDSENRDLKTVKEKDDILFEDLQDNVNENGEGEIEDEEEEGYDDDDDDWDWDEGVGKLAKGYVWNGGSNPQANRQTSDSSSAKMSTPADKVLRKFENKINLDKLNVTDSVINKVTEKSRQKEADMYRIKDKADRATVEQVLDPRTRMILFKMLTRGIITEINGCISTGKEANVYHASTANGESRAIKIYKTSILVFKDRDKYVSGEFRFRHGYCKGNPRKMVKTWAEKEMRNLIRLNTAEIPCPEPIMLRSHVLVMSFIGKDDMPAPLLKNVQLSESKARELYLQVIQYMRRMYQDARLVHADLSEFNMLYHGGGVYIIDVSQSVEHDHPHALEFLRKDCANVNDFFMRHSVAVMTVRELFEFVTDPSITHENMDAYLSKAMEIASQRTKEERSSQDHVDEEVFKRAYIPRTLNEVKNYERDMDIIMKLKEEDMAMNAQQDNILYQTVTGLKKDLSGVQKVPALLENQVEERTCSDSEDIGSSECSDTDSEEQGDHARPKKHTTDPDIDKKERKKMVKEAQREKRKNKIPKHVKKRKEKTAKTKKGK",
# ST32A_HUMAN
"MGANTSRKPPVFDENEDVNFDHFEILRAIGKGSFGKVCIVQKNDTKKMYAMKYMNKQKCVERNEVRNVFKELQIMQGLEHPFLVNLWYSFQDEEDMFMVVDLLLGGDLRYHLQQNVHFKEETVKLFICELVMALDYLQNQRIIHRDMKPDNILLDEHGHVHITDFNIAAMLPRETQITTMAGTKPYMAPEMFSSRKGAGYSFAVDWWSLGVTAYELLRGRRPYHIRSSTSSKEIVHTFETTVVTYPSAWSQEMVSLLKKLLEPNPDQRFSQLSDVQNFPYMNDINWDAVFQKRLIPGFIPNKGRLNCDPTFELEEMILESKPLHKKKKRLAKKEKDMRKCDSSQTCLLQEHLDSVQKEFIIFNREKVNRDFNKRQPNLALEQTKDPQGEDGQNNNL",
# SNRK_HUMAN
"MAGFKRGYDGKIAGLYDLDKTLGRGHFAVVKLARHVFTGEKVAVKVIDKTKLDTLATGHLFQEVRCMKLVQHPNIVRLYEVIDTQTKLYLILELGDGGDMFDYIMKHEEGLNEDLAKKYFAQIVHAISYCHKLHVVHRDLKPENVVFFEKQGLVKLTDFGFSNKFQPGKKLTTSCGSLAYSAPEILLGDEYDAPAVDIWSLGVILFMLVCGQPPFQEANDSETLTMIMDCKYTVPSHVSKECKDLITRMLQRDPKRRASLEEIENHPWLQGVDPSPATKYNIPLVSYKNLSEEEHNSIIQRMVLGDIADRDAIVEALETNRYNHITATYFLLAERILREKQEKEIQTRSASPSNIKAQFRQSWPTKIDVPQDLEDDLTATPLSHATVPQSPARAADSVLNGHRSKGLCDSAKKDDLPELAGPALSTVPPASLKPTASGRKCLFRVEEDEEEDEEDKKPMSLSTQVVLRRKPSVTNRLTSRKSAPVLNQIFEEGESDDEFDMDENLPPKLSRLKMNIASPGTVHKRYHRRKSQGRGSSCSSSETSDDDSESRRRLDKDSGFTYSWHRRDSSEGPPGSEGDGGGQSKPSNASGGVDKASPSENNAGGGSPSSGSGGNPTNTSGTTRRCAGPSNSMQLASRSAGELVESLKLMSLCLGSQLHGSTKYIIDPQNGLSFSSVKVQEKSTWKMCISSTGNAGQVPAVGGIKFFSDHMADTTTELERIKSKNLKNNVLQLPLCEKTISVNIQRNPKEGLLCASSPASCCHVI",
# EPHAA_HUMAN
"METCAGPHPLRLFLCRMQLCLALLLGPWRPGTAEEVILLDSKASQAELGWTALPSNGWEEISGVDEHDRPIRTYQVCNVLEPNQDNWLQTGWISRGRGQRIFVELQFTLRDCSSIPGAAGTCKETFNVYYLETEADLGRGRPRLGGSRPRKIDTIAADESFTQGDLGERKMKLNTEVREIGPLSRRGFHLAFQDVGACVALVSVRVYYKQCRATVRGLATFPATAAESAFSTLVEVAGTCVAHSEGEPGSPPRMHCGADGEWLVPVGRCSCSAGFQERGDFCEACPPGFYKVSPRRPLCSPCPEHSRALENASTFCVCQDSYARSPTDPPSASCTRPPSAPRDLQYSLSRSPLVLRLRWLPPADSGGRSDVTYSLLCLRCGREGPAGACEPCGPRVAFLPRQAGLRERAATLLHLRPGARYTVRVAALNGVSGPAAAAGTTYAQVTVSTGPGAPWEEDEIRRDRVEPQSVSLSWREPIPAGAPGANDTEYEIRYYEKGQSEQTYSMVKTGAPTVTVTNLKPATRYVFQIRAASPGPSWEAQSFNPSIEVQTLGEAASGSRDQSPAIVVTVVTISALLVLGSVMSVLAIWRRPCSYGKGGGDAHDEEELYFHFKVPTRRTFLDPQSCGDLLQAVHLFAKELDAKSVTLERSLGGGRFGELCCGCLQLPGRQELLVAVHMLRDSASDSQRLGFLAEALTLGQFDHSHIVRLEGVVTRGSTLMIVTEYMSHGALDGFLRRHEGQLVAGQLMGLLPGLASAMKYLSEMGYVHRGLAARHVLVSSDLVCKISGFGRGPRDRSEAVYTTMSGRSPALWAAPETLQFGHFSSASDVWSFGIIMWEVMAFGERPYWDMSGQDVIKAVEDGFRLPPPRNCPNLLHRLMLDCWQKDPGERPRFSQIHSILSKMVQDPEPPKCALTTCPRPPTPLADRAFSTFPSFGSVGAWLEALDLCRYKDSFAAAGYGSLEAVAEMTAQDLVSLGISLAEHREALLSGISALQARVLQLQGQGVQV",
# SG196_HUMAN
"MEKQPQNSRRGLAPREVPPAVGLLLIMALMNTLLYLCLDHFFIAPRQSTVDPTHCPYGHFRIGQMKNCSPWLSCEELRTEVRQLKRVGEGAVKRVFLSEWKEHKVALSQLTSLEMKDDFLHGLQMLKSLQGTHVVTLLGYCEDDNTMLTEYHPLGSLSNLEETLNLSKYQNVNTWQHRLELAMDYVSIINYLHHSPVGTRVMCDSNDLPKTLSQYLLTSNFSILANDLDALPLVNHSSGMLVKCGHRELHGDFVAPEQLWPYGEDVPFHDDLMPSYDEKIDIWKIPDISSFLLGHIEGSDMVRFHLFDIHKACKSQTPSERPTAQDVLETYQKVLDTLRDAMMSQAREML",
# MYO3A_HUMAN
"MFPLIGKTIIFDNFPDPSDTWEITETIGKGTYGKVFKVLNKKNGQKAAVKILDPIHDIDEEIEAEYNILKALSDHPNVVRFYGIYFKKDKVNGDKLWLVLELCSGGSVTDLVKGFLKRGERMSEPLIAYILHEALMGLQHLHNNKTIHRDVKGNNILLTTEGGVKLVDFGVSAQLTSTRHRRNTSVGTPFWMAPEVIACEQQLDTTYDARCDTWSLGITAIELGDGDPPLADLHPMRALFKIPRNPPPKLRQPELWSAEFNDFISKCLTKDYEKRPTVSELLQHKFITQIEGKDVMLQKQLTEFIGIHQCMGGTEKARRERIHTKKGNFNRPLISNLKDVDDLATLEILDENTVSEQLEKCYSRDQIYVYVGDILIALNPFQSLGLYSTKHSKLYIGSKRTASPPHIFAMADLGYQSMITYNSDQCIVISGESGAGKTENAHLLVQQLTVLGKANNRTLQEKILQVNNLVEAFGNACTIINDNSSRFGKYLEMKFTSSGAVVGAQISEYLLEKSRVIHQAIGEKNFHIFYYIYAGLAEKKKLAHYKLPENKPPRYLQNDHLRTVQDIMNNSFYKSQYELIEQCFKVIGFTMEQLGSIYSILAAILNVGNIEFSSVATEHQIDKSHISNHTALENCASLLCIRADELQEALTSHCVVTRGETIIRPNTVEKATDVRDAMAKTLYGRLFSWIVNCINSLLKHDSSPSGNGDELSIGILDIFGFENFKKNSFEQLCINIANEQIQYYYNQHVFAWEQNEYLNEDVDARVIEYEDNWPLLDMFLQKPMGLLSLLDEESRFPKATDQTLVEKFEGNLKSQYFWRPKRMELSFGIHHYAGKVLYNASGFLAKNRDTLPTDIVLLLRSSDNSVIRQLVNHPLTKTGNLPHSKTKNVINYQMRTSEKLINLAKGDTGEATRHARETTNMKTQTVASYFRYSLMDLLSKMVVGQPHFVRCIKPNSERQARKYDKEKVLLQLRYTGILETARIRRLGFSHRILFANFIKRYYLLCYKSSEEPRMSPDTCATILEKAGLDNWALGKTKVFLKYYHVEQLNLMRKEAIDKLILIQACVRAFLCSRRYQKIQEKRKESAIIIQSAARGHLVRKQRKEIVDMKNTAVTTIQTSDQEFDYKKNFENTRESFVKKQAENAISANERFISAPNNKGSVSVVKTSTFKPEEETTNAVESNNRVYQTPKKMNNVYEEEVKQEFYLVGPEVSPKQKSVKDLEENSNLRKVEKEEAMIQSYYQRYTEERNCEESKAAYLERKAISERPSYPVPWLAENETSFKKTLEPTLSQRSIYQNANSMEKEKKTSVVTQRAPICSQEEGRGRLRHETVKERQVEPVTQAQEEEDKAAVFIQSKYRGYKRRQQLRKDKMSSFKHQRIVTTPTEVARNTHNLYSYPTKHEEINNIKKKDNKDSKATSEREACGLAIFSKQISKLSEEYFILQKKLNEMILSQQLKSLYLGVSHHKPINRRVSSQQCLSGVCKGEEPKILRPPRRPRKPKTLNNPEDSTYYYLLHKSIQEEKRRPRKDSQGKLLDLEDFYYKEFLPSRSGPKEHSPSLRERRPQQELQNQCIKANERCWAAESPEKEEEREPAANPYDFRRLLRKTSQRRRLVQQS",
# WNK3_HUMAN
"MATDSGDPASTEDSEKPDGISFENRVPQVAATLTVEARLKEKNSTFSASGETVERKRFFRKSVEMTEDDKVAESSPKDERIKAAMNIPRVDKLPSNVLRGGQEVKYEQCSKSTSEISKDCFKEKNEKEMEEEAEMKAVATSPSGRFLKFDIELGRGAFKTVYKGLDTETWVEVAWCELQDRKLTKAEQQRFKEEAEMLKGLQHPNIVRFYDSWESILKGKKCIVLVTELMTSGTLKTYLKRFKVMKPKVLRSWCRQILKGLQFLHTRTPPIIHRDLKCDNIFITGPTGSVKIGDLGLATLMRTSFAKSVIGTPEFMAPEMYEEHYDESVDVYAFGMCMLEMATSEYPYSECQNAAQIYRKVTSGIKPASFNKVTDPEVKEIIEGCIRQNKSERLSIRDLLNHAFFAEDTGLRVELAEEDDCSNSSLALRLWVEDPKKLKGKHKDNEAIEFSFNLETDTPEEVAYEMVKSGFFHESDSKAVAKSIRDRVTPIKKTREKKPAGCLEERRDSQCKSMGNVFPQPQNTTLPLAPAQQTGAECEETEVDQHVRQQLLQRKPQQHCSSVTGDNLSEAGAASVIHSDTSSQPSVAYSSNQTMGSQMVSNIPQAEVNVPGQIYSSQQLVGHYQQVSGLQKHSKLTQPQILPLVQGQSTVLPVHVLGPTVVSQPQVSPLTVQKVPQIKPVSQPVGAEQQAALLKPDLVRSLNQDVATTKENVSSPDNPSGNGKQDRIKQRRASCPRPEKGTKFQLTVLQVSTSGDNMVECQLETHNNKMVTFKFDVDGDAPEDIADYMVEDNFVLESEKEKFVEELRAIVGQAQEILHVHFATERATGVDSITVDSNSSQTGSSEQVQINSTSTQTSNESAPQSSPVGRWRFCINQTIRNRETQSPPSLQHSMSAVPGRHPLPSPKNTSNKEISRDTLLTIENNPCHRALFTSKSEHKDVVDGKISECASVETKQPAILYQVEDNRQIMAPVTNSSSYSTTSVRAVPAECEGLTKQASIFIPVYPCHQTASQADALMSHPGESTQTSGNSLTTLAFDQKPQTLSVQQPAMDAEFISQEGETTVNTEASSPKTVIPTQTPGLEPTTLQPTTVLESDGERPPKLEFADNRIKTLDEKLRNLLYQEHSISSIYPESQKDTQSIDSPFSSSAEDTLSCPVTEVIAISHCGIKDSPVQSPNFQQTGSKLLSNVAASQPANISVFKRDLNVITSVPSELCLHEMSSDASLPGDPEAYPAAVSSGGAIHLQTGGGYFGLSFTCPSLKNPISKKSWTRKLKSWAYRLRQSTSFFKRSKVRQVETEEMRSAIAPDPIPLTRESTADTRALNRCKAMSGSFQRGRFQVITIPQQQSAKMTSFGIEHISVFSETNHSSEEAFIKTAKSQLVEIEPATQNPKTSFSYEKLQALQETCKENKGVPKQGDNFLSFSAACETDVSSVTPEKEFEETSATGSSMQSGSELLLKEREILTAGKQPSSDSEFSASLAGSGKSVAKTGPESNQCLPHHEEQAYAQTQSSLFYSPSSPMSSDDESEIEDEDLKVELQRLREKHIQEVVNLQTQQNKELQELYERLRSIKDSKTQSTEIPLPPASPRRPRSFKSKLRSRPQSLTHVDNGIVATGKSCLINELENPLCVESNAASCQQSPASKKGMFTDDLHKLVDDWTKEAVGNSLIKPSLNQLKQSQHKLETENWNKVSENTPSTMGYTSTWISSLSQIRGAVPTSLPQGLSLPSFPGPLSSYGMPHVCQYNAVAGAGYPVQWVGISGTTQQSVVIPAQSGGPFQPGMNMQAFPTSSVQNPATIPPGPK",
# PRAG1_HUMAN
"MHQTLCLNPESLKMSACSDFVEHIWKPGSCKNCFCLRSDHQLVAGPPQPRAGSLPPPPRLPPRPENCRLEDEGVNSSPYSKPTIAVKPTMMSSEASDVWTEANLSAEVSQVIWRRAPGKLPLPKQEDAPVVYLGSFRGVQKPAGPSTSPDGNSRCPPAYTMVGLHNLEPRGERNIAFHPVSFPEEKAVHKEKPSFPYQDRPSTQESFRQKLAAFAGTTSGCHQGPGPLRESLPSEDDSDQRCSPSGDSEGGEYCSILDCCPGSPVAKAASQTAGSRGRHGGRDCSPTCWEQGKCSGPAEQEKRGPSFPKECCSQGPTAHPSCLGPKKLSLTSEAAISSDGLSCGSGSGSGSGASSPFVPHLESDYCSLMKEPAPEKQQDPGCPGVTPSRCLGLTGEPQPPAHPREATQPEPIYAESTKRKKAAPVPSKSQAKIEHAAAAQGQGQVCTGNAWAQKAASGWGRDSPDPTPQVSATITVMAAHPEEDHRTIYLSSPDSAVGVQWPRGPVSQNSEVGEEETSAGQGLSSRESHAHSASESKPKERPAIPPKLSKSSPVGSPVSPSAGGPPVSPLADLSDGSSGGSSIGPQPPSQGPADPAPSCRTNGVAISDPSRCPQPAASSASEQRRPRFQAGTWSRQCRIEEEEEVEQELLSHSWGRETKNGPTDHSNSTTWHRLHPTDGSSGQNSKVGTGMSKSASFAFEFPKDRSGIETFSPPPPPPKSRHLLKMNKSSSDLEKVSQGSAESLSPSFRGVHVSFTTGSTDSLASDSRTCSDGGPSSELAHSPTNSGKKLFAPVPFPSGSTEDVSPSGPQQPPPLPQKKIVSRAASSPDGFFWTQGSPKPGTASPKLNLSHSETNVHDESHFSYSLSPGNRHHPVFSSSDPLEKAFKGSGHWLPAAGLAGNRGGCGSPGLQCKGAPSASSSQLSVSSQASTGSTQLQLHGLLSNISSKEGTYAKLGGLYTQSLARLVAKCEDLFMGGQKKELHFNENNWSLFKLTCNKPCCDSGDAIYYCATCSEDPGSTYAVKICKAPEPKTVSYCSPSVPVHFNIQQDCGHFVASVPSSMLSSPDAPKDPVPALPTHPPAQEQDCVVVITREVPHQTASDFVRDSAASHQAEPEAYERRVCFLLLQLCNGLEHLKEHGIIHRDLCLENLLLVHCTLQAGPGPAPAPAPAPAPAAAAPPCSSAAPPAGGTLSPAAGPASPEGPREKQLPRLIISNFLKAKQKPGGTPNLQQKKSQARLAPEIVSASQYRKFDEFQTGILIYELLHQPNPFEVRAQLRERDYRQEDLPPLPALSLYSPGLQQLAHLLLEADPIKRIRIGEAKRVLQCLLWGPRRELVQQPGTSEEALCGTLHNWIDMKRALMMMKFAEKAVDRRRGVELEDWLCCQYLASAEPGALLQSLKLLQLL",
# NEK10_HUMAN
"MPDQDKKVKTTEKSTDKQQEITIRDYSDLKRLRCLLNVQSSKQQLPAINFDSAQNSMTKSEPAIRAGGHRARGQWHESTEAVELENFSINYKNERNFSKHPQRKLFQEIFTALVKNRLISREWVNRAPSIHFLRVLICLRLLMRDPCYQEILHSLGGIENLAQYMEIVANEYLGYGEEQHTVDKLVNMTYIFQKLAAVKDQREWVTTSGAHKTLVNLLGARDTNVLLGSLLALASLAESQECREKISELNIVENLLMILHEYDLLSKRLTAELLRLLCAEPQVKEQVKLYEGIPVLLSLLHSDHLKLLWSIVWILVQVCEDPETSVEIRIWGGIKQLLHILQGDRNFVSDHSSIGSLSSANAAGRIQQLHLSEDLSPREIQENTFSLQAACCAALTELVLNDTNAHQVVQENGVYTIAKLILPNKQKNAAKSNLLQCYAFRALRFLFSMERNRPLFKRLFPTDLFEIFIDIGHYVRDISAYEELVSKLNLLVEDELKQIAENIESINQNKAPLKYIGNYAILDHLGSGAFGCVYKVRKHSGQNLLAMKEVNLHNPAFGKDKKDRDSSVRNIVSELTIIKEQLYHPNIVRYYKTFLENDRLYIVMELIEGAPLGEHFSSLKEKHHHFTEERLWKIFIQLCLALRYLHKEKRIVHRDLTPNNIMLGDKDKVTVTDFGLAKQKQENSKLTSVVGTILYSCPEVLKSEPYGEKADVWAVGCILYQMATLSPPFYSTNMLSLATKIVEAVYEPVPEGIYSEKVTDTISRCLTPDAEARPDIVEVSSMISDVMMKYLDNLSTSQLSLEKKLERERRRTQRYFMEANRNTVTCHHELAVLSHETFEKASLSSSSSGAASLKSELSESADLPPEGFQASYGKDEDRACDEILSDDNFNLENAEKDTYSEVDDELDISDNSSSSSSSPLKESTFNILKRSFSASGGERQSQTRDFTGGTGSRPRPALLPLDLLLKVPPHMLRAHIKEIEAELVTGWQSHSLPAVILRNLKDHGPQMGTFLWQASAGIAVSQRKVRQISDPIQQILIQLHKIIYITQLPPALHHNLKRRVIERFKKSLFSQQSNPCNLKSEIKKLSQGSPEPIEPNFFTADYHLLHRSSGGNSLSPNDPTGLPTSIELEEGITYEQMQTVIEEVLEESGYYNFTSNRYHSYPWGTKNHPTKR",
# EPHA6_HUMAN
"MGGCEVREFLLQFGFFLPLLTAWPGDCSHVSNNQVVLLDTTTVLGELGWKTYPLNGWDAITEMDEHNRPIHTYQVCNVMEPNQNNWLRTNWISRDAAQKIYVEMKFTLRDCNSIPWVLGTCKETFNLFYMESDESHGIKFKPNQYTKIDTIAADESFTQMDLGDRILKLNTEIREVGPIERKGFYLAFQDIGACIALVSVRVFYKKCPFTVRNLAMFPDTIPRVDSSSLVEVRGSCVKSAEERDTPKLYCGADGDWLVPLGRCICSTGYEEIEGSCHACRPGFYKAFAGNTKCSKCPPHSLTYMEATSVCQCEKGYFRAEKDPPSMACTRPPSAPRNVVFNINETALILEWSPPSDTGGRKDLTYSVICKKCGLDTSQCEDCGGGLRFIPRHTGLINNSVIVLDFVSHVNYTFEIEAMNGVSELSFSPKPFTAITVTTDQDAPSLIGVVRKDWASQNSIALSWQAPAFSNGAILDYEIKYYEKEHEQLTYSSTRSKAPSVIITGLKPATKYVFHIRVRTATGYSGYSQKFEFETGDETSDMAAEQGQILVIATAAVGGFTLLVILTLFFLITGRCQWYIKAKMKSEEKRRNHLQNGHLRFPGIKTYIDPDTYEDPSLAVHEFAKEIDPSRIRIERVIGAGEFGEVCSGRLKTPGKREIPVAIKTLKGGHMDRQRRDFLREASIMGQFDHPNIIRLEGVVTKRSFPAIGVEAFCPSFLRAGFLNSIQAPHPVPGGGSLPPRIPAGRPVMIVVEYMENGSLDSFLRKHDGHFTVIQLVGMLRGIASGMKYLSDMGYVHRDLAARNILVNSNLVCKVSDFGLSRVLEDDPEAAYTTTGGKIPIRWTAPEAIAYRKFSSASDAWSYGIVMWEVMSYGERPYWEMSNQDVILSIEEGYRLPAPMGCPASLHQLMLHCWQKERNHRPKFTDIVSFLDKLIRNPSALHTLVEDILVMPESPGEVPEYPLFVTVGDWLDSIKMGQYKNNFVAAGFTTFDLISRMSIDDIRRIGVILIGHQRRIVSSIQTLRLHMMHIQEKGFHV",
# KC1G1_HUMAN
"MDHPSREKDERQRTTKPMAQRSAHCSRPSGSSSSSGVLMVGPNFRVGKKIGCGNFGELRLGKNLYTNEYVAIKLEPIKSRAPQLHLEYRFYKQLGSAGEGLPQVYYFGPCGKYNAMVLELLGPSLEDLFDLCDRTFTLKTVLMIAIQLLSRMEYVHSKNLIYRDVKPENFLIGRQGNKKEHVIHIIDFGLAKEYIDPETKKHIPYREHKSLTGTARYMSINTHLGKEQSRRDDLEALGHMFMYFLRGSLPWQGLKADTLKERYQKIGDTKRNTPIEALCENFPEEMATYLRYVRRLDFFEKPDYEYLRTLFTDLFEKKGYTFDYAYDWVGRPIPTPVGSVHVDSGASAITRESHTHRDRPSQQQPLRNQVVSSTNGELNVDDPTGAHSNAPITAHAEVEVVEEAKCCCFFKRKRKKTAQRHK",
# PEAK1_HUMAN
"MSACNTFTEHVWKPGECKNCFKPKSLHQLPPDPEKAPITHGNVKTNANHSNNHRIRNTGNFRPPVAKKPTIAVKPTMIVADGQSICGELSIQEHCENKPVIIGWNRNRAALSQKPLNNNNEDDEGISHVPKPYGNNDSAKKMSDNNNGLTEVLKEIAGLDTAPQIRGNETNSRETFLGRINDCYKRSLERKLPPSCMIGGIKETQGKHVILSGSTEVISNEGGRFCYPEFSSGEESEEDVLFSNMEEEHESWDESDEELLAMEIRMRGQPRFANFRANTLSPVRFFVDKKWNTIPLRNKSLQRICAVDYDDSYDEILNGYEENSVVSYGQGSIQSMVSSDSTSPDSSLTEESRSETASSLSQKICNGGLSPGNPGDSKDMKEIEPNYESPSSNNQDKDSSQASKSSIKVPETHKAVLALRLEEKDGKIAVQTEKEESKASTDVAGQAVTINLVPTEEQAKPYRVVNLEQPLCKPYTVVDVSAAMASEHLEGPVNSPKTKSSSSTPNSPVTSSSLTPGQISAHFQKSSAIRYQEVWTSSTSPRQKIPKVELITSGTGPNVPPRKNCHKSAPTSPTATNISSKTIPVKSPNLSEIKFNSYNNAGMPPFPIIIHDEPTYARSSKNAIKVPIVINPNAYDNLAIYKSFLGTSGELSVKEKTTSVISHTYEEIETESKVPDNTTSKTTDCLQTKGFSNSTEHKRGSVAQKVQEFNNCLNRGQSSPQRSYSSSHSSPAKIQRATQEPVAKIEGTQESQMVGSSSTREKASTVLSQIVASIQPPQSPPETPQSGPKACSVEELYAIPPDADVAKSTPKSTPVRPKSLFTSQPSGEAEAPQTTDSPTTKVQKDPSIKPVTPSPSKLVTSPQSEPPAPFPPPRSTSSPYHAGNLLQRHFTNWTKPTSPTRSTEAESVLHSEGSRRAADAKPKRWISFKSFFRRRKTDEEDDKEKEREKGKLVGLDGTVIHMLPPPPVQRHHWFTEAKGESSEKPAIVFMYRCDPAQGQLSVDQSKARTDQAAVMEKGRAENALLQDSEKKRSHSSPSQIPKKILSHMTHEVTEDFSPRDPRTVVGKQDGRGCTSVTTALSLPELEREDGKEDISDPMDPNPCSATYSNLGQSRAAMIPPKQPRQPKGAVDDAIAFGGKTDQEAPNASQPTPPPLPKKMIIRANTEPISKDLQKSMESSLCVMANPTYDIDPNWDASSAGSSISYELKGLDIESYDSLERPLRKERPVPSAANSISSLTTLSIKDRFSNSMESLSSRRGPSCRQGRGIQKPQRQALYRGLENREEVVGKIRSLHTDALKKLAVKCEDLFMAGQKDQLRFGVDSWSDFRLTSDKPCCEAGDAVYYTASYAKDPLNNYAVKICKSKAKESQQYYHSLAVRQSLAVHFNIQQDCGHFLAEVPNRLLPWEDPDDPEKDEDDMEETEEDAKGETDGKNPKPCSEAASSQKENQGVMSKKQRSHVVVITREVPCLTVADFVRDSLAQHGKSPDLYERQVCLLLLQLCSGLEHLKPYHVTHCDLRLENLLLVHYQPGGTAQGFGPAEPSPTSSYPTRLIVSNFSQAKQKSHLVDPEILRDQSRLAPEIITATQYKKCDEFQTGILIYEMLHLPNPFDENPELKEREYTRADLPRIPFRSPYSRGLQQLASCLLNPNPSERILISDAKGILQCLLWGPREDLFQTFTACPSLVQRNTLLQNWLDIKRTLLMIKFAEKSLDREGGISLEDWLCAQYLAFATTDSLSCIVKILQHR",
# SBK1_HUMAN
"MSVGCPEPEPPRSLTCCGPGTAPGPGAGVPLLTEDMQALTLRTLAASDVTKHYELVRELGKGTYGKVDLVVYKGTGTKMALKFVNKSKTKLKNFLREVSITNSLSSSPFIIKVFDVVFETEDCYVFAQEYAPAGDLFDIIPPQVGLPEDTVKRCVQQLGLALDFMHGRQLVHRDIKPENVLLFDRECRRVKLADFGMTRRVGCRVKRVSGTIPYTAPEVCQAGRADGLAVDTGVDVWAFGVLIFCVLTGNFPWEAASGADAFFEEFVRWQRGRLPGLPSQWRRFTEPALRMFQRLLALEPERRGPAKEVFRFLKHELTSELRRRPSHRARKPPGDRPPAAGPLRLEAPGPLKRTVLTESGSGSRPAPPAVGSVPLPVPVPVPVPVPVPVPEPGLAPQGPPGRTDGRADKSKGQVVLATAIEICV",
# STK31_HUMAN
"MWVQGHSSRASATESVSFSGIVQMDEDTHYDKVEDVVGSHIEDAVTFWAQSINRNKDIMKIGCSLSEVCPQASSVLGNLDPNKIYGGLFSEDQCWYRCKVLKIISVEKCLVRYIDYGNTEILNRSDIVEIPLELQFSSVAKKYKLWGLHIPSDQEVTQFDQGTTFLGSLIFEKEIKMRIKATSEDGTVIAQAEYGSVDIGEEVLKKGFAEKCRLASRTDICEEKKLDPGQLVLRNLKSPIPLWGHRSNQSTFSRPKGHLSEKMTLDLKDENDAGNLITFPKESLAVGDFNLGSNVSLEKIKQDQKLIEENEKLKTEKDALLESYKALELKVEQIAQELQQEKAAAVDLTNHLEYTLKTYIDTRMKNLAAKMEILKEMRHVDISVRFGKDLSDAIQVLDEGCFTTPASLNGLEIIWAEYSLAQENIKTCEYVSEGNILIAQRNEMQQKLYMSVEDFILEVDESSLNKRLKTLQDLSVSLEAVYGQAKEGANSDEILKKFYDWKCDKREEFTSVRSETDASLHRLVAWFQRTLKVFDLSVEGSLISEDAMDNIDEILEKTESSVCKELEIALVDQGDADKEIISNTYSQVLQKIHSEERLIATVQAKYKDSIEFKKQLIEYLNKSPSVDHLLSIKKTLKSLKALLRWKLVEKSNLEESDDPDGSQIEKIKEEITQLRNNVFQEIYHEREEYEMLTSLAQKWFPELPLLHPEIGLLKYMNSGGLLTMSLERDLLDAEPMKELSSKRPLVRSEVNGQIILLKGYSVDVDTEAKVIERAATYHRAWREAEGDSGLLPLIFLFLCKSDPMAYLMVPYYPRANLNAVQANMPLNSEETLKVMKGVAQGLHTLHKADIIHGSLHQNNVFALNREQGIVGDFDFTKSVSQRASVNMMVGDLSLMSPELKMGKPASPGSDLYAYGCLLLWLSVQNQEFEINKDGIPKVDQFHLDDKVKSLLCSLICYRSSMTAEQVLNAECFLMPKEQSVPNPEKDTEYTLYKKEEEIKTENLDKCMEKTRNGEANFDC",
# ANKK1_HUMAN
"MAADPTELRLGSLPVFTRDDFEGDWRLVASGGFSQVFQARHRRWRTEYAIKCAPCLPPDAASSDVNYLIEEAAKMKKIKFQHIVSIYGVCKQPLGIVMEFMANGSLEKVLSTHSLCWKLRFRIIHETSLAMNFLHSIKPPLLHLDLKPGNILLDSNMHVKISDFGLSKWMEQSTRMQYIERSALRGMLSYIPPEMFLESNKAPGPKYDVYSFAIVIWELLTQKKPYSGFNMMMIIIRVAAGMRPSLQPVSDQWPSEAQQMVDLMKRCWDQDPKKRPCFLDITIETDILLSLLQSRVAVPESKALARKVSCKLSLRQPGEVNEDISQELMDSDSGNYLKRALQLSDRKNLVPRDEELCIYENKVTPLHFLVAQGSVEQVRLLLAHEVDVDCQTASGYTPLLIAAQDQQPDLCALLLAHGADANRVDEDGWAPLHFAAQNGDDGTARLLLDHGACVDAQEREGWTPLHLAAQNNFENVARLLVSRQADPNLHEAEGKTPLHVAAYFGHVSLVKLLTSQGAELDAQQRNLRTPLHLAVERGKVRAIQHLLKSGAVPDALDQSGYGPLHTAAARGKYLICKMLLRYGASLELPTHQGWTPLHLAAYKGHLEIIHLLAESHANMGALGAVNWTPLHLAARHGEEAVVSALLQCGADPNAAEQSGWTPLHLAVQRSTFLSVINLLEHHANVHARNKVGWTPAHLAALKGNTAILKVLVEAGAQLDVQDGVSCTPLQLALRSRKQGIMSFLEGKEPSVATLGGSKPGAEMEI",
# UHMK1_HUMAN
"MAGSGCAWGAEPPRFLEAFGRLWQVQSRLGSGSSASVYRVRCCGNPGSPPGALKQFLPPGTTGAAASAAEYGFRKERAALEQLQGHRNIVTLYGVFTIHFSPNVPSRCLLLELLDVSVSELLLYSSHQGCSMWMIQHCARDVLEALAFLHHEGYVHADLKPRNILWSAENECFKLIDFGLSFKEGNQDVKYIQTDGYRAPEAELQNCLAQAGLQSDTECTSAVDLWSLGIILLEMFSGMKLKHTVRSQEWKANSSAIIDHIFASKAVVNAAIPAYHLRDLIKSMLHDDPSRRIPAEMALCSPFFSIPFAPHIEDLVMLPTPVLRLLNVLDDDYLENEEEYEDVVEDVKEECQKYGPVVSLLVPKENPGRGQVFVEYANAGDSKAAQKLLTGRMFDGKFVVATFYPLSAYKRGYLYQTLL",
# KCC1B_HUMAN
"MLLLKKHTEDISSVYEIRERLGSGAFSEVVLAQERGSAHLVALKCIPKKALRGKEALVENEIAVLRRISHPNIVALEDVHESPSHLYLAMELVTGGELFDRIMERGSYTEKDASHLVGQVLGAVSYLHSLGIVHRDLKPENLLYATPFEDSKIMVSDFGLSKIQAGNMLGTACGTPGYVAPELLEQKPYGKAVDVWALGVISYILLCGYPPFYDESDPELFSQILRASYEFDSPFWDDISESAKDFIRHLLERDPQKRFTCQQALRHLWISGDTAFDRDILGSVSEQIRKNFARTHWKRAFNATSFLRHIRKLGQIPEGEGASEQGMARHSHSGLRAGQPPKW",
# TBCK_HUMAN
"MFPLKDAEMGAFTFFASALPHDVCGSNGLPLTPNSIKILGRFQILKTITHPRLCQYVDISRGKHERLVVVAEHCERSLEDLLRERKPVSCSTVLCIAFEVLQGLQYMNKHGIVHRALSPHNILLDRKGHIKLAKFGLYHMTAHGDDVDFPIGYPSYLAPEVIAQGIFKTTDHMPSKKPLPSGPKSDVWSLGIILFELCVGRKLFQSLDISERLKFLLTLDCVDDTLIVLAEEHGCLDIIKELPETVIDLLNKCLTFHPSKRPTPDQLMKDKVFSEVSPLYTPFTKPASLFSSSLRCADLTLPEDISQLCKDINNDYLAERSIEEVYYLWCLAGGDLEKELVNKEIIRSKPPICTLPNFLFEDGESFGQGRDRSSLLDDTTVTLSLCQLRNRLKDVGGEAFYPLLEDDQSNLPHSNSNNELSAAATLPLIIREKDTEYQLNRIILFDRLLKAYPYKKNQIWKEARVDIPPLMRGLTWAALLGVEGAIHAKYDAIDKDTPIPTDRQIEVDIPRCHQYDELLSSPEGHAKFRRVLKAWVVSHPDLVYWQGLDSLCAPFLYLNFNNEALAYACMSAFIPKYLYNFFLKDNSHVIQEYLTVFSQMIAFHDPELSNHLNEIGFIPDLYAIPWFLTMFTHVFPLHKIFHLWDTLLLGNSSFPFCIGVAILQQLRDRLLANGFNECILLFSDLPEIDIERCVRESINLFCWTPKSATYRQHAQPPKPSSDSSGGRSSAPYFSAECPDPPKTDLSRESIPLNDLKSEVSPRISAEDLIDLCELTVTGHFKTPSKKTKSSKPKLLVVDIRNSEDFIRGHISGSINIPFSAAFTAEGELTQGPYTAMLQNFKGKVIVIVGHVAKHTAEFAAHLVKMKYPRICILDGGINKIKPTGLLTIPSPQI",
# SMG1_HUMAN
"MSRRAPGSRLSSGGGGGGTKYPRSWNDWQPRTDSASADPDNLKYSSSRDRGGSSSYGLQPSNSAVVSRQRHDDTRVHADIQNDEKGGYSVNGGSGENTYGRKSLGQELRVNNVTSPEFTSVQHGSRALATKDMRKSQERSMSYSDESRLSNLLRRITREDDRDRRLATVKQLKEFIQQPENKLVLVKQLDNILAAVHDVLNESSKLLQELRQEGACCLGLLCASLSYEAEKIFKWIFSKFSSSAKDEVKLLYLCATYKALETVGEKKAFSSVMQLVMTSLQSILENVDTPELLCKCVKCILLVARCYPHIFSTNFRDTVDILVGWHIDHTQKPSLTQQVSGWLQSLEPFWVADLAFSTTLLGQFLEDMEAYAEDLSHVASGESVDEDVPPPSVSLPKLAALLRVFSTVVRSIGERFSPIRGPPITEAYVTDVLYRVMRCVTAANQVFFSEAVLTAANECVGVLLGSLDPSMTIHCDMVITYGLDQLENCQTCGTDYIISVLNLLTLIVEQINTKLPSSFVEKLFIPSSKLLFLRYHKEKEVVAVAHAVYQAVLSLKNIPVLETAYKLILGEMTCALNNLLHSLQLPEACSEIKHEAFKNHVFNVDNAKFVVIFDLSALTTIGNAKNSLIGMWALSPTVFALLSKNLMIVHSDLAVHFPAIQYAVLYTLYSHCTRHDHFISSSLSSSSPSLFDGAVISTVTTATKKHFSIILNLLGILLKKDNLNQDTRKLLMTWALEAAVLMKKSETYAPLFSLPSFHKFCKGLLANTLVEDVNICLQACSSLHALSSSLPDDLLQRCVDVCRVQLVHSGTRIRQAFGKLLKSIPLDVVLSNNNHTEIQEISLALRSHMSKAPSNTFHPQDFSDVISFILYGNSHRTGKDNWLERLFYSCQRLDKRDQSTIPRNLLKTDAVLWQWAIWEAAQFTVLSKLRTPLGRAQDTFQTIEGIIRSLAAHTLNPDQDVSQWTTADNDEGHGNNQLRLVLLLQYLENLEKLMYNAYEGCANALTSPPKVIRTFFYTNRQTCQDWLTRIRLSIMRVGLLAGQPAVTVRHGFDLLTEMKTTSLSQGNELEVTIMMVVEALCELHCPEAIQGIAVWSSSIVGKNLLWINSVAQQAEGRFEKASVEYQEHLCAMTGVDCCISSFDKSVLTLANAGRNSASPKHSLNGESRKTVLSKPTDSSPEVINYLGNKACECYISIADWAAVQEWQNAIHDLKKSTSSTSLNLKADFNYIKSLSSFESGKFVECTEQLELLPGENINLLAGGSKEKIDMKKLLPNMLSPDPRELQKSIEVQLLRSSVCLATALNPIEQDQKWQSITENVVKYLKQTSRIAIGPLRLSTLTVSQSLPVLSTLQLYCSSALENTVSNRLSTEDCLIPLFSEALRSCKQHDVRPWMQALRYTMYQNQLLEKIKEQTVPIRSHLMELGLTAAKFARKRGNVSLATRLLAQCSEVQLGKTTTAQDLVQHFKKLSTQGQVDEKWGPELDIEKTKLLYTAGQSTHAMEMLSSCAISFCKSVKAEYAVAKSILTLAKWIQAEWKEISGQLKQVYRAQHQQNFTGLSTLSKNILTLIELPSVNTMEEEYPRIESESTVHIGVGEPDFILGQLYHLSSVQAPEVAKSWAALASWAYRWGRKVVDNASQGEGVRLLPREKSEVQNLLPDTITEEEKERIYGILGQAVCRPAGIQDEDITLQITESEDNEEDDMVDVIWRQLISSCPWLSELDESATEGVIKVWRKVVDRIFSLYKLSCSAYFTFLKLNAGQIPLDEDDPRLHLSHRVEQSTDDMIVMATLRLLRLLVKHAGELRQYLEHGLETTPTAPWRGIIPQLFSRLNHPEVYVRQSICNLLCRVAQDSPHLILYPAIVGTISLSSESQASGNKFSTAIPTLLGNIQGEELLVSECEGGSPPASQDSNKDEPKSGLNEDQAMMQDCYSKIVDKLSSANPTMVLQVQMLVAELRRVTVLWDELWLGVLLQQHMYVLRRIQQLEDEVKRVQNNNTLRKEEKIAIMREKHTALMKPIVFALEHVRSITAAPAETPHEKWFQDNYGDAIENALEKLKTPLNPAKPGSSWIPFKEIMLSLQQRAQKRASYILRLEEISPWLAAMTNTEIALPGEVSARDTVTIHSVGGTITILPTKTKPKKLLFLGSDGKSYPYLFKGLEDLHLDERIMQFLSIVNTMFATINRQETPRFHARHYSVTPLGTRSGLIQWVDGATPLFGLYKRWQQREAALQAQKAQDSYQTPQNPGIVPRPSELYYSKIGPALKTVGLSLDVSRRDWPLHVMKAVLEELMEATPPNLLAKELWSSCTTPDEWWRVTQSYARSTAVMSMVGYIIGLGDRHLDNVLIDMTTGEVVHIDYNVCFEKGKSLRVPEKVPFRMTQNIETALGVTGVEGVFRLSCEQVLHIMRRGRETLLTLLEAFVYDPLVDWTAGGEAGFAGAVYGGGGQQAESKQSKREMEREITRSLFSSRVAEIKVNWFKNRDEMLVVLPKLDGSLDEYLSLQEQLTDVEKLQGKLLEEIEFLEGAEGVDHPSHTLQHRYSEHTQLQTQQRAVQEAIQVKLNEFEQWITHYQAAFNNLEATQLASLLQEISTQMDLGPPSYVPATAFLQNAGQAHLISQCEQLEGEVGALLQQRRSVLRGCLEQLHHYATVALQYPKAIFQKHRIEQWKTWMEELICNTTVERCQELYRKYEMQYAPQPPPTVCQFITATEMTLQRYAADINSRLIRQVERLKQEAVTVPVCEDQLKEIERCIKVFLHENGEEGSLSLASVIISALCTLTRRNLMMEGAASSAGEQLVDLTSRDGAWFLEELCSMSGNVTCLVQLLKQCHLVPQDLDIPNPMEASETVHLANGVYTSLQELNSNFRQIIFPEALRCLMKGEYTLESMLHELDGLIEQTTDGVPLQTLVESLQAYLRNAAMGLEEETHAHYIDVARLLHAQYGELIQPRNGSVDETPKMSAGQMLLVAFDGMFAQVETAFSLLVEKLNKMEIPIAWRKIDIIREARSTQVNFFDDDNHRQVLEEIFFLKRLQTIKEFFRLCGTFSKTLSGSSSLEDQNTVNGPVQIVNVKTLFRNSCFSEDQMAKPIKAFTADFVRQLLIGLPNQALGLTLCSFISALGVDIIAQVEAKDFGAESKVSVDDLCKKAVEHNIQIGKFSQLVMNRATVLASSYDTAWKKHDLVRRLETSISSCKTSLQRVQLHIAMFQWQHEDLLINRPQAMSVTPPPRSAILTSMKKKLHTLSQIETSIATVQEKLAALESSIEQRLKWAGGANPALAPVLQDFEATIAERRNLVLKESQRASQVTFLCSNIIHFESLRTRTAEALNLDAALFELIKRCQQMCSFASQFNSSVSELELRLLQRVDTGLEHPIGSSEWLLSAHKQLTQDMSTQRAIQTEKEQQIETVCETIQNLVDNIKTVLTGHNRQLGDVKHLLKAMAKDEEAALADGEDVPYENSVRQFLGEYKSWQDNIQTVLFTLVQAMGQVRSQEHVEMLQEITPTLKELKTQSQSIYNNLVSFASPLVTDATNECSSPTSSATYQPSFAAAVRSNTGQKTQPDVMSQNARKLIQKNLATSADTPPSTVPGTGKSVACSPKKAVRDPKTGKAVQERNSYAVSVWKRVKAKLEGRDVDPNRRMSVAEQVDYVIKEATNLDNLAQLYEGWTAWV",
# MYLK2_HUMAN
"MATENGAVELGIQNPSTDKAPKGPTGERPLAAGKDPGPPDPKKAPDPPTLKKDAKAPASEKGDGTLAQPSTSSQGPKGEGDRGGGPAEGSAGPPAALPQQTATPETSVKKPKAEQGASGSQDPGKPRVGKKAAEGQAAARRGSPAFLHSPSCPAIISSSEKLLAKKPPSEASELTFEGVPMTHSPTDPRPAKAEEGKNILAESQKEVGEKTPGQAGQAKMQGDTSRGIEFQAVPSEKSEVGQALCLTAREEDCFQILDDCPPPPAPFPHRMVELRTGNVSSEFSMNSKEALGGGKFGAVCTCMEKATGLKLAAKVIKKQTPKDKEMVLLEIEVMNQLNHRNLIQLYAAIETPHEIVLFMEYIEGGELFERIVDEDYHLTEVDTMVFVRQICDGILFMHKMRVLHLDLKPENILCVNTTGHLVKIIDFGLARRYNPNEKLKVNFGTPEFLSPEVVNYDQISDKTDMWSMGVITYMLLSGLSPFLGDDDTETLNNVLSGNWYFDEETFEAVSDEAKDFVSNLIVKDQRARMNAAQCLAHPWLNNLAEKAKRCNRRLKSQILLKKYLMKRRWKKNFIAVSAANRFKKISSSGALMALGV",
# M3K15_HUMAN
"MESGGGNAPAGALGAASESPQCPPPPGVEGAAGPAEPDGAAEGAAGGSGEGESGGGPRRALRAVYVRSESSQGGAAGGPEAGARQCLLRACEAEGAHLTSVPFGELDFGETAVLDAFYDADVAVVDMSDVSRQPSLFYHLGVRESFDMANNVILYHDTDADTALSLKDMVTQKNTASSGNYYFIPYIVTPCADYFCCESDAQRRASEYMQPNWDNILGPLCMPLVDRFISLLKDIHVTSCVYYKETLLNDIRKAREKYQGEELAKELARIKLRMDNTEVLTSDIIINLLLSYRDIQDYDAMVKLVETLEMLPTCDLADQHNIKFHYAFALNRRNSTGDREKALQIMLQVLQSCDHPGPDMFCLCGRIYKDIFLDSDCKDDTSRDSAIEWYRKGFELQSSLYSGINLAVLLIVAGQQFETSLELRKIGVRLNSLLGRKGSLEKMNNYWDVGQFFSVSMLAHDVGKAVQAAERLFKLKPPVWYLRSLVQNLLLIRRFKKTIIEHSPRQERLNFWLDIIFEATNEVTNGLRFPVLVIEPTKVYQPSYVSINNEAEERTVSLWHVSPTEMKQMHEWNFTASSIKGISLSKFDERCCFLYVHDNSDDFQIYFSTEEQCSRFFSLVKEMITNTAGSTVELEGETDGDTLEYEYDHDANGERVVLGKGTYGIVYAGRDLSNQVRIAIKEIPERDSRYSQPLHEEIALHKYLKHRNIVQYLGSVSENGYIKIFMEQVPGGSLSALLRSKWGPMKEPTIKFYTKQILEGLKYLHENQIVHRDIKGDNVLVNTYSGVVKISDFGTSKRLAGVNPCTETFTGTLQYMAPEIIDQGPRGYGAPADIWSLGCTIIEMATSKPPFHELGEPQAAMFKVGMFKIHPEIPEALSAEARAFILSCFEPDPHKRATTAELLREGFLRQVNKGKKNRIAFKPSEGPRGVVLALPTQGEPMATSSSEHGSVSPDSDAQPDALFERTRAPRHHLGHLLSVPDESSALEDRGLASSPEDRDQGLFLLRKDSERRAILYKILWEEQNQVASNLQECVAQSSEELHLSVGHIKQIIGILRDFIRSPEHRVMATTISKLKVDLDFDSSSISQIHLVLFGFQDAVNKILRNHLIRPHWMFAMDNIIRRAVQAAVTILIPELRAHFEPTCETEGVDKDMDEAEEGYPPATGPGQEAQPHQQHLSLQLGELRQETNRLLEHLVEKEREYQNLLRQTLEQKTQELYHLQLKLKSNCITENPAGPYGQRTDKELIDWLRLQGADAKTIEKIVEEGYTLSDILNEITKEDLRYLRLRGGLLCRLWSAVSQYRRAQEASETKDKA",
# LRRK2_HUMAN
"MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFTYSERASKLFQGKNIHVPLLIVLDSYMRVASVQQVGWSLLCKLIEVCPGTMQSLMGPQDVGNDWEVLGVHQLILKMLTVHNASVNLSVIGLKTLDLLLTSGKITLLILDEESDIFMLIFDAMHSFPANDEVQKLGCKALHVLFERVSEEQLTEFVENKDYMILLSALTNFKDEEEIVLHVLHCLHSLAIPCNNVEVLMSGNVRCYNIVVEAMKAFPMSERIQEVSCCLLHRLTLGNFFNILVLNEVHEFVVKAVQQYPENAALQISALSCLALLTETIFLNQDLEEKNENQENDDEGEEDKLFWLEACYKALTWHRKNKHVQEAACWALNNLLMYQNSLHEKIGDEDGHFPAHREVMLSMLMHSSSKEVFQASANALSTLLEQNVNFRKILLSKGIHLNVLELMQKHIHSPEVAESGCKMLNHLFEGSNTSLDIMAAVVPKILTVMKRHETSLPVQLEALRAILHFIVPGMPEESREDTEFHHKLNMVKKQCFKNDIHKLVLAALNRFIGNPGIQKCGLKVISSIVHFPDALEMLSLEGAMDSVLHTLQMYPDDQEIQCLGLSLIGYLITKKNVFIGTGHLLAKILVSSLYRFKDVAEIQTKGFQTILAILKLSASFSKLLVHHSFDLVIFHQMSSNIMEQKDQQFLNLCCKCFAKVAMDDYLKNVMLERACDQNNSIMVECLLLLGADANQAKEGSSLICQVCEKESSPKLVELLLNSGSREQDVRKALTISIGKGDSQIISLLLRRLALDVANNSICLGGFCIGKVEPSWLGPLFPDKTSNLRKQTNIASTLARMVIRYQMKSAVEEGTASGSDGNFSEDVLSKFDEWTFIPDSSMDSVFAQSDDLDSEGSEGSFLVKKKSNSISVGEFYRDAVLQRCSPNLQRHSNSLGPIFDHEDLLKRKRKILSSDDSLRSSKLQSHMRHSDSISSLASEREYITSLDLSANELRDIDALSQKCCISVHLEHLEKLELHQNALTSFPQQLCETLKSLTHLDLHSNKFTSFPSYLLKMSCIANLDVSRNDIGPSVVLDPTVKCPTLKQFNLSYNQLSFVPENLTDVVEKLEQLILEGNKISGICSPLRLKELKILNLSKNHISSLSENFLEACPKVESFSARMNFLAAMPFLPPSMTILKLSQNKFSCIPEAILNLPHLRSLDMSSNDIQYLPGPAHWKSLNLRELLFSHNQISILDLSEKAYLWSRVEKLHLSHNKLKEIPPEIGCLENLTSLDVSYNLELRSFPNEMGKLSKIWDLPLDELHLNFDFKHIGCKAKDIIRFLQQRLKKAVPYNRMKLMIVGNTGSGKTTLLQQLMKTKKSDLGMQSATVGIDVKDWPIQIRDKRKRDLVLNVWDFAGREEFYSTHPHFMTQRALYLAVYDLSKGQAEVDAMKPWLFNIKARASSSPVILVGTHLDVSDEKQRKACMSKITKELLNKRGFPAIRDYHFVNATEESDALAKLRKTIINESLNFKIRDQLVVGQLIPDCYVELEKIILSERKNVPIEFPVIDRKRLLQLVRENQLQLDENELPHAVHFLNESGVLLHFQDPALQLSDLYFVEPKWLCKIMAQILTVKVEGCPKHPKGIISRRDVEKFLSKKRKFPKNYMSQYFKLLEKFQIALPIGEEYLLVPSSLSDHRPVIELPHCENSEIIIRLYEMPYFPMGFWSRLINRLLEISPYMLSGRERALRPNRMYWRQGIYLNWSPEAYCLVGSEVLDNHPESFLKITVPSCRKGCILLGQVVDHIDSLMEEWFPGLLEIDICGEGETLLKKWALYSFNDGEEHQKILLDDLMKKAEEGDLLVNPDQPRLTIPISQIAPDLILADLPRNIMLNNDELEFEQAPEFLLGDGSFGSVYRAAYEGEEVAVKIFNKHTSLRLLRQELVVLCHLHHPSLISLLAAGIRPRMLVMELASKGSLDRLLQQDKASLTRTLQHRIALHVADGLRYLHSAMIIYRDLKPHNVLLFTLYPNAAIIAKIADYGIAQYCCRMGIKTSEGTPGFRAPEVARGNVIYNQQADVYSFGLLLYDILTTGGRIVEGLKFPNEFDELEIQGKLPDPVKEYGCAPWPMVEKLIKQCLKENPQERPTSAQVFDILNSAELVCLTRRILLPKNVIVECMVATHHNSRNASIWLGCGHTDRGQLSFLDLNTEGYTSEEVADSRILCLALVHLPVEKESWIVSGTQSGTLLVINTEDGKKRHTLEKMTDSVTCLYCNSFSKQSKQKNFLLVGTADGKLAIFEDKTVKLKGAAPLKILNIGNVSTPLMCLSESTNSTERNVMWGGCGTKIFSFSNDFTIQKLIETRTSQLFSYAAFSDSNIITVVVDTALYIAKQNSPVVEVWDKKTEKLCGLIDCVHFLREVMVKENKESKHKMSYSGRVKTLCLQKNTALWIGTGGGHILLLDLSTRRLIRVIYNFCNSVRVMMTAQLGSLKNVMLVLGYNRKNTEGTQKQKEIQSCLTVWDINLPHEVQNLEKHIEVRKELAEKMRRTSVE",
# M3K21_HUMAN
"MALRGAAGATDTPVSSAGGAPGGSASSSSTSSGGSASAGAGLWAALYDYEARGEDELSLRRGQLVEVLSQDAAVSGDEGWWAGQVQRRLGIFPANYVAPCRPAASPAPPPSRPSSPVHVAFERLELKELIGAGGFGQVYRATWQGQEVAVKAARQDPEQDAAAAAESVRREARLFAMLRHPNIIELRGVCLQQPHLCLVLEFARGGALNRALAAANAAPDPRAPGPRRARRIPPHVLVNWAVQIARGMLYLHEEAFVPILHRDLKSSNILLLEKIEHDDICNKTLKITDFGLAREWHRTTKMSTAGTYAWMAPEVIKSSLFSKGSDIWSYGVLLWELLTGEVPYRGIDGLAVAYGVAVNKLTLPIPSTCPEPFAKLMKECWQQDPHIRPSFALILEQLTAIEGAVMTEMPQESFHSMQDDWKLEIQQMFDELRTKEKELRSREEELTRAALQQKSQEELLKRREQQLAEREIDVLERELNILIFQLNQEKPKVKKRKGKFKRSRLKLKDGHRISLPSDFQHKITVQASPNLDKRRSLNSSSSSPPSSPTMMPRLRAIQLTSDESNKTWGRNTVFRQEEFEDVKRNFKKKGCTWGPNSIQMKDRTDCKERIRPLSDGNSPWSTILIKNQKTMPLASLFVDQPGSCEEPKLSPDGLEHRKPKQIKLPSQAYIDLPLGKDAQRENPAEAESWEEAASANAATVSIEMTPTNSLSRSPQRKKTESALYGCTVLLASVALGLDLRELHKAQAAEEPLPKEEKKKREGIFQRASKSRRSASPPTSLPSTCGEASSPPSLPLSSALGILSTPSFSTKCLLQMDSEDPLVDSAPVTCDSEMLTPDFCPTAPGSGREPALMPRLDTDCSVSRNLPSSFLQQTCGNVPYCASSKHRPSHHRRTMSDGNPTPTGATIISATGASALPLCPSPAPHSHLPREVSPKKHSTVHIVPQRRPASLRSRSDLPQAYPQTAVSQLAQTACVVGRPGPHPTQFLAAKERTKSHVPSLLDADVEGQSRDYTVPLCRMRSKTSRPSIYELEKEFLS",
# HASP_HUMAN
"MAASLPGPGSRLFRTYGAADGRRQRRPGREAAQWFPPQDRRRFFNSSGSSDASIGDPSQSDDPDDPDDPDFPGSPVRRRRRRPGGRVPKDRPSLTVTPKRWKLRARPSLTVTPRRLGLRARPPQKCSTPCGPLRLPPFPSRDSGRLSPDLSVCGQPRDGDELGISASLFSSLASPCPGSPTPRDSVISIGTSACLVAASAVPSGLHLPEVSLDRASLPCSQEEATGGAKDTRMVHQTRASLRSVLFGLMNSGTPEDSEFRADGKNMRESCCKRKLVVGNGPEGPGLSSTGKRRATGQDSCQERGLQEAVRREHQEASVPKGRIVPRGIDRLERTRSSRKSKHQEATETSLLHSHRFKKGQKLGKDSFPTQDLTPLQNVCFWTKTRASFSFHKKKIVTDVSEVCSIYTTATSLSGSLLSECSNRPVMNRTSGAPSSWHSSSMYLLSPLNTLSISNKKASDAEKVYGECSQKGPVPFSHCLPTEKLQRCEKIGEGVFGEVFQTIADHTPVAIKIIAIEGPDLVNGSHQKTFEEILPEIIISKELSLLSGEVCNRTEGFIGLNSVHCVQGSYPPLLLKAWDHYNSTKGSANDRPDFFKDDQLFIVLEFEFGGIDLEQMRTKLSSLATAKSILHQLTASLAVAEASLRFEHRDLHWGNVLLKKTSLKKLHYTLNGKSSTIPSCGLQVSIIDYTLSRLERDGIVVFCDVSMDEDLFTGDGDYQFDIYRLMKKENNNRWGEYHPYSNVLWLHYLTDKMLKQMTFKTKCNTPAMKQIKRKIQEFHRTMLNFSSATDLLCQHSLFK",
# TRIB3_HUMAN
"MRATPLAAPAGSLSRKKRLELDDNLDTERPVQKRARSGPQPRLPPCLLPLSPPTAPDRATAVATASRLGPYVLLEPEEGGRAYQALHCPTGTEYTCKVYPVQEALAVLEPYARLPPHKHVARPTEVLAGTQLLYAFFTRTHGDMHSLVRSRHRIPEPEAAVLFRQMATALAHCHQHGLVLRDLKLCRFVFADRERKKLVLENLEDSCVLTGPDDSLWDKHACPAYVGPEILSSRASYSGKAADVWSLGVALFTMLAGHYPFQDSEPVLLFGKIRRGAYALPAGLSAPARCLVRCLLRREPAERLTATGILLHPWLRQDPMPLAPTRSHLWEAAQVVPDGLGLDEAREEEGDREVVLYG",
# CTRO_HUMAN
"MLKFKYGARNPLDAGAAEPIASRASRLNLFFQGKPPFMTQQQMSPLSREGILDALFVLFEECSQPALMKIKHVSNFVRKYSDTIAELQELQPSAKDFEVRSLVGCGHFAEVQVVREKATGDIYAMKVMKKKALLAQEQVSFFEEERNILSRSTSPWIPQLQYAFQDKNHLYLVMEYQPGGDLLSLLNRYEDQLDENLIQFYLAELILAVHSVHLMGYVHRDIKPENILVDRTGHIKLVDFGSAAKMNSNKMVNAKLPIGTPDYMAPEVLTVMNGDGKGTYGLDCDWWSVGVIAYEMIYGRSPFAEGTSARTFNNIMNFQRFLKFPDDPKVSSDFLDLIQSLLCGQKERLKFEGLCCHPFFSKIDWNNIRNSPPPFVPTLKSDDDTSNFDEPEKNSWVSSSPCQLSPSGFSGEELPFVGFSYSKALGILGRSESVVSGLDSPAKTSSMEKKLLIKSKELQDSQDKCHKMEQEMTRLHRRVSEVEAVLSQKEVELKASETQRSLLEQDLATYITECSSLKRSLEQARMEVSQEDDKALQLLHDIREQSRKLQEIKEQEYQAQVEEMRLMMNQLEEDLVSARRRSDLYESELRESRLAAEEFKRKATECQHKLLKAKDQGKPEVGEYAKLEKINAEQQLKIQELQEKLEKAVKASTEATELLQNIRQAKERAERELEKLQNREDSSEGIRKKLVEAEELEEKHREAQVSAQHLEVHLKQKEQHYEEKIKVLDNQIKKDLADKETLENMMQRHEEEAHEKGKILSEQKAMINAMDSKIRSLEQRIVELSEANKLAANSSLFTQRNMKAQEEMISELRQQKFYLETQAGKLEAQNRKLEEQLEKISHQDHSDKNRLLELETRLREVSLEHEEQKLELKRQLTELQLSLQERESQLTALQAARAALESQLRQAKTELEETTAEAEEEIQALTAHRDEIQRKFDALRNSCTVITDLEEQLNQLTEDNAELNNQNFYLSKQLDEASGANDEIVQLRSEVDHLRREITEREMQLTSQKQTMEALKTTCTMLEEQVMDLEALNDELLEKERQWEAWRSVLGDEKSQFECRVRELQRMLDTEKQSRARADQRITESRQVVELAVKEHKAEILALQQALKEQKLKAESLSDKLNDLEKKHAMLEMNARSLQQKLETERELKQRLLEEQAKLQQQMDLQKNHIFRLTQGLQEALDRADLLKTERSDLEYQLENIQVLYSHEKVKMEGTISQQTKLIDFLQAKMDQPAKKKKGLFSRRKEDPALPTQVPLQYNELKLALEKEKARCAELEEALQKTRIELRSAREEAAHRKATDHPHPSTPATARQQIAMSAIVRSPEHQPSAMSLLAPPSSRRKESSTPEEFSRRLKERMHHNIPHRFNVGLNMRATKCAVCLDTVHFGRQASKCLECQVMCHPKCSTCLPATCGLPAEYATHFTEAFCRDKMNSPGLQTKEPSSSLHLEGWMKVPRNNKRGQQGWDRKYIVLEGSKVLIYDNEAREAGQRPVEEFELCLPDGDVSIHGAVGASELANTAKADVPYILKMESHPHTTCWPGRTLYLLAPSFPDKQRWVTALESVVAGGRVSREKAEADAKLLGNSLLKLEGDDRLDMNCTLPFSDQVVLVGTEEGLYALNVLKNSLTHVPGIGAVFQIYIIKDLEKLLMIAGEERALCLVDVKKVKQSLAQSHLPAQPDISPNIFEAVKGCHLFGAGKIENGLCICAAMPSKVVILRYNENLSKYCIRKEIETSEPCSCIHFTNYSILIGTNKFYEIDMKQYTLEEFLDKNDHSLAPAVFAASSNSFPVSIVQVNSAGQREEYLLCFHEFGVFVDSYGRRSRTDDLKWSRLPLAFAYREPYLFVTHFNSLEVIEIQARSSAGTPARAYLDIPNPRYLGPAISSGAIYLASSYQDKLRVICCKGNLVKESGTEHHRGPSTSRSSPNKRGPPTYNEHITKRVASSPAPPEGPSHPREPSTPHRYREGRTELRRDKSPGRPLEREKSPGRMLSTRRERSPGRLFEDSSRGRLPAGAVRTPLSQVNKVWDQSSV",
# KKCC1_HUMAN
"MEGGPAVCCQDPRAELVERVAAIDVTHLEEADGGPEPTRNGVDPPPRARAASVIPGSTSRLLPARPSLSARKLSLQERPAGSYLEAQAGPYATGPASHISPRAWRRPTIESHHVAISDAEDCVQLNQYKLQSEIGKGAYGVVRLAYNESEDRHYAMKVLSKKKLLKQYGFPRRPPPRGSQAAQGGPAKQLLPLERVYQEIAILKKLDHVNVVKLIEVLDDPAEDNLYLVFDLLRKGPVMEVPCDKPFSEEQARLYLRDVILGLEYLHCQKIVHRDIKPSNLLLGDDGHVKIADFGVSNQFEGNDAQLSSTAGTPAFMAPEAISDSGQSFSGKALDVWATGVTLYCFVYGKCPFIDDFILALHRKIKNEPVVFPEEPEISEELKDLILKMLDKNPETRIGVPDIKLHPWVTKNGEEPLPSEEEHCSVVEVTEEEVKNSVRLIPSWTTVILVKSMLRKRSFGNPFEPQARREERSMSAPGNLLVKEGFGEGGKSPELPGVQEDEAAS",
# LRRK1_HUMAN
"MAGMSQRPPSMYWCVGPEESAVCPERAMETLNGAGDTGGKPSTRGGDPAARSRRTEGIRAAYRRGDRGGARDLLEEACDQCASQLEKGQLLSIPAAYGDLEMVRYLLSKRLVELPTEPTDDNPAVVAAYFGHTAVVQELLESLPGPCSPQRLLNWMLALACQRGHLGVVKLLVLTHGADPESYAVRKNEFPVIVRLPLYAAIKSGNEDIAIFLLRHGAYFCSYILLDSPDPSKHLLRKYFIEASPLPSSYPGKTALRVKWSHLRLPWVDLDWLIDISCQITELDLSANCLATLPSVIPWGLINLRKLNLSDNHLGELPGVQSSDEIICSRLLEIDISSNKLSHLPPGFLHLSKLQKLTASKNCLEKLFEEENATNWIGLRKLQELDISDNKLTELPALFLHSFKSLNSLNVSRNNLKVFPDPWACPLKCCKASRNALECLPDKMAVFWKNHLKDVDFSENALKEVPLGLFQLDALMFLRLQGNQLAALPPQEKWTCRQLKTLDLSRNQLGKNEDGLKTKRIAFFTTRGRQRSGTEAASVLEFPAFLSESLEVLCLNDNHLDTVPPSVCLLKSLSELYLGNNPGLRELPPELGQLGNLWQLDTEDLTISNVPAEIQKEGPKAMLSYLRAQLRKAEKCKLMKMIIVGPPRQGKSTLLEILQTGRAPQVVHGEATIRTTKWELQRPAGSRAKVESVEFNVWDIGGPASMATVNQCFFTDKALYVVVWNLALGEEAVANLQFWLLNIEAKAPNAVVLVVGTHLDLIEAKFRVERIATLRAYVLALCRSPSGSRATGFPDITFKHLHEISCKSLEGQEGLRQLIFHVTCSMKDVGSTIGCQRLAGRLIPRSYLSLQEAVLAEQQRRSRDDDVQYLTDRQLEQLVEQTPDNDIKDYEDLQSAISFLIETGTLLHFPDTSHGLRNLYFLDPIWLSECLQRIFNIKGSRSVAKNGVIRAEDLRMLLVGTGFTQQTEEQYFQFLAKFEIALPVANDSYLLPHLLPSKPGLDTHGMRHPTANTIQRVFKMSFVPVGFWQRFIARMLISLAEMDLQLFENKKNTKSRNRKVTIYSFTGNQRNRCSTFRVKRNQTIYWQEGLLVTFDGGYLSVESSDVNWKKKKSGGMKIVCQSEVRDFSAMAFITDHVNSLIDQWFPALTATESDGTPLMEQYVPCPVCETAWAQHTDPSEKSEDVQYFDMEDCVLTAIERDFISCPRHPDLPVPLQELVPELFMTDFPARLFLENSKLEHSEDEGSVLGQGGSGTVIYRARYQGQPVAVKRFHIKKFKNFANVPADTMLRHLRATDAMKNFSEFRQEASMLHALQHPCIVALIGISIHPLCFALELAPLSSLNTVLSENARDSSFIPLGHMLTQKIAYQIASGLAYLHKKNIIFCDLKSDNILVWSLDVKEHINIKLSDYGISRQSFHEGALGVEGTPGYQAPEIRPRIVYDEKVDMFSYGMVLYELLSGQRPALGHHQLQIAKKLSKGIRPVLGQPEEVQFRRLQALMMECWDTKPEKRPLALSVVSQMKDPTFATFMYELCCGKQTAFFSSQGQEYTVVFWDGKEESRNYTVVNTEKGLMEVQRMCCPGMKVSCQLQVQRSLWTATEDQKIYIYTLKGMCPLNTPQQALDTPAVVTCFLAVPVIKKNSYLVLAGLADGLVAVFPVVRGTPKDSCSYLCSHTANRSKFSIADEDARQNPYPVKAMEVVNSGSEVWYSNGPGLLVIDCASLEICRRLEPYMAPSMVTSVVCSSEGRGEEVVWCLDDKANSLVMYHSTTYQLCARYFCGVPSPLRDMFPVRPLDTEPPAASHTANPKVPEGDSIADVSIMYSEELGTQILIHQESLTDYCSMSSYSSSPPRQAARSPSSLPSSPASSSSVPFSTDCEDSDMLHTPGAASDRSEHDLTPMDGETFSQHLQAVKILAVRDLIWVPRRGGDVIVIGLEKDSGAQRGRVIAVLKARELTPHGVLVDAAVVAKDTVVCTFENENTEWCLAVWRGWGAREFDIFYQSYEELGRLEACTRKRR",
# TEX14_HUMAN
"MSRAVRLPVPCPVQLGTLRNDSLEAQLHEYVKQGNYVKVKKILKKGIYVDAVNSLGQTALFVAALLGLRKFVDVLVDYGSDPNHRCFDGSTPVHAAAFSGNQWILSKLLDAGGDLRLHDERGQNPKTWALTAGKERSTQIVEFMQRCASHMQAIIQGFSYDLLKKIDSPQRLVYSPSWCGGLVQGNPNGSPNRLLKAGVISAQNIYSFGFGKAMPWFQFYLTGATQMAYLGSLPVIGEKEVIQADDEPTFSFFSGPYMVMTNLVWNGSRVTVKELNLPTHPHCSRLRLADLLIAEQEHSSKLRHPYLLQLMAVCLSQDLEKTRLVYERITIGTLFSVLHERRSQFPVLHMEVIVHLLLQISDALRYLHFQGFIHRSLSSYAVHIISPGEARLTNLEYMLESEDRGVQRDLTRVPLPTQLYNWAAPEVILQKAATVKSDIYSFSMIMQEILTDDIPWKGLDGSVVKKAVVSGNYLEADVRLPKPYYDIVKSGIHVKQKDRTMNLQDIRYILKNDLKDFTGAQRTQPTESPRVQRYGLHPDVNVYLGLTSEHPRETPDMEIIELKEMGSQPHSPRVHSLFTEGTLDPQAPDPCLMARETQNQDAPCPAPFMAEEASSPSTGQPSLCSFEINEIYSGCLILEDDIEEPPGAASSLEADGPNQVDELKSMEEELDKMEREACCFGSEDESSSKAETEYSFDDWDWQNGSLSSLSLPESTREAKSNLNNMSTTEEYLISKCVLDLKIMQTIMHENDDRLRNIEQILDEVEMKQKEQEERMSLWATSREFTNAYKLPLAVGPPSLNYIPPVLQLSGGQKPDTSGNYPTLPRFPRMLPTLCDPGKQNTDEQFQCTQGAKDSLETSRIQNTSSQGRPRESTAQAKATQFNSALFTLSSHRQGPSASPSCHWDSTRMSVEPVSSEIYNAESRNKDDGKVHLKWKMEVKEMAKKAATGQLTVPPWHPQSSLTLESEAENEPDALLQPPIRSPENTDWQRVIEYHRENDEPRGNGKFDKTGNNDCDSDQHGRQPRLGSFTSIRHPSPRQKEQPEHSEAFQASSDTLVAVEKSYSHQSMQSTCSPESSEDITDEFLTPDGEYFYSSTAQENLALETSSPIEEDFEGIQGAFAQPQVSGEEKFQMRKILGKNAEILPRSQFQPVRSTEDEQEETSKESPKELKEKDISLTDIQDLSSISYEPDSSFKEASCKTPKINHAPTSVSTPLSPGSVSSAASQYKDCLESITFQVKTEFASCWNSQEFIQTLSDDFISVRERAKKLDSLLTSSETPPSRLTGLKRLSSFIGAGSPSLVKACDSSPPHATQRRSLPKVEAFSQHHIDELPPPSQELLDDIELLKQQQGSSTVLHENTASDGGGTANDQRHLEEQETDSKKEDSSMLLSKETEDLGEDTERAHSTLDEDLERWLQPPEESVELQDLPKGSERETNIKDQKVGEEKRKREDSITPERRKSEGVLGTSEEDELKSCFWKRLGWSESSRIIVLDQSDLSD",
# MAST4_HUMAN
"MGEKVSEAPEPVPRGCSGHGSRTPASALVAASSPGASSAESSSGSETLSEEGEPGGFSREHQPPPPPPLGGTLGARAPAAWAPASVLLERGVLALPPPLPGGAVPPAPRGSSASQEEQDEELDHILSPPPMPFRKCSNPDVASGPGKSLKYKRQLSEDGRQLRRGSLGGALTGRYLLPNPVAGQAWPASAETSNLVRMRSQALGQSAPSLTASLKELSLPRRGSFCRTSNRKSLIGNGQSPALPRPHSPLSAHAGNSPQDSPRNFSPSASAHFSFARRTDGRRWSLASLPSSGYGTNTPSSTVSSSCSSQEKLHQLPYQPTPDELHFLSKHFCTTESIATENRCRNTPMRPRSRSLSPGRSPACCDHEIIMMNHVYKERFPKATAQMEERLKEIITSYSPDNVLPLADGVLSFTHHQIIELARDCLDKSHQGLITSRYFLELQHKLDKLLQEAHDRSESGELAFIKQLVRKILIVIARPARLLECLEFDPEEFYYLLEAAEGHAKEGQGIKTDIPRYIISQLGLNKDPLEEMAHLGNYDSGTAETPETDESVSSSNASLKLRRKPRESDFETIKLISNGAYGAVYFVRHKESRQRFAMKKINKQNLILRNQIQQAFVERDILTFAENPFVVSMYCSFETRRHLCMVMEYVEGGDCATLMKNMGPLPVDMARMYFAETVLALEYLHNYGIVHRDLKPDNLLVTSMGHIKLTDFGLSKVGLMSMTTNLYEGHIEKDAREFLDKQVCGTPEYIAPEVILRQGYGKPVDWWAMGIILYEFLVGCVPFFGDTPEELFGQVISDEINWPEKDEAPPPDAQDLITLLLRQNPLERLGTGGAYEVKQHRFFRSLDWNSLLRQKAEFIPQLESEDDTSYFDTRSEKYHHMETEEEDDTNDEDFNVEIRQFSSCSHRFSKVFSSIDRITQNSAEEKEDSVDKTKSTTLPSTETLSWSSEYSEMQQLSTSNSSDTESNRHKLSSGLLPKLAISTEGEQDEAASCPGDPHEEPGKPALPPEECAQEEPEVTTPASTISSSTLSVGSFSEHLDQINGRSECVDSTDNSSKPSSEPASHMARQRLESTEKKKISGKVTKSLSASALSLMIPGDMFAVSPLGSPMSPHSLSSDPSSSRDSSPSRDSSAASASPHQPIVIHSSGKNYGFTIRAIRVYVGDSDIYTVHHIVWNVEEGSPACQAGLKAGDLITHINGEPVHGLVHTEVIELLLKSGNKVSITTTPFENTSIKTGPARRNSYKSRMVRRSKKSKKKESLERRRSLFKKLAKQPSPLLHTSRSFSCLNRSLSSGESLPGSPTHSLSPRSPTPSYRSTPDFPSGTNSSQSSSPSSSAPNSPAGSGHIRPSTLHGLAPKLGGQRYRSGRRKSAGNIPLSPLARTPSPTPQPTSPQRSPSPLLGHSLGNSKIAQAFPSKMHSPPTIVRHIVRPKSAEPPRSPLLKRVQSEEKLSPSYGSDKKHLCSRKHSLEVTQEEVQREQSQREAPLQSLDENVCDVPPLSRARPVEQGCLKRPVSRKVGRQESVDDLDRDKLKAKVVVKKADGFPEKQESHQKSHGPGSDLENFALFKLEEREKKVYPKAVERSSTFENKASMQEAPPLGSLLKDALHKQASVRASEGAMSDGRVPAEHRQGGGDFRRAPAPGTLQDGLCHSLDRGISGKGEGTEKSSQAKELLRCEKLDSKLANIDYLRKKMSLEDKEDNLCPVLKPKMTAGSHECLPGNPVRPTGGQQEPPPASESRAFVSSTHAAQMSAVSFVPLKALTGRVDSGTEKPGLVAPESPVRKSPSEYKLEGRSVSCLKPIEGTLDIALLSGPQASKTELPSPESAQSPSPSGDVRASVPPVLPSSSGKKNDTTSARELSPSSLKMNKSYLLEPWFLPPSRGLQNSPAVSLPDPEFKRDRKGPHPTARSPGTVMESNPQQREGSSPKHQDHTTDPKLLTCLGQNLHSPDLARPRCPLPPEASPSREKPGLRESSERGPPTARSERSAARADTCREPSMELCFPETAKTSDNSKNLLSVGRTHPDFYTQTQAMEKAWAPGGKTNHKDGPGEARPPPRDNSSLHSAGIPCEKELGKVRRGVEPKPEALLARRSLQPPGIESEKSEKLSSFPSLQKDGAKEPERKEQPLQRHPSSIPPPPLTAKDLSSPAARQHCSSPSHASGREPGAKPSTAEPSSSPQDPPKPVAAHSESSSHKPRPGPDPGPPKTKHPDRSLSSQKPSVGATKGKEPATQSLGGSSREGKGHSKSGPDVFPATPGSQNKASDGIGQGEGGPSVPLHTDRAPLDAKPQPTSGGRPLEVLEKPVHLPRPGHPGPSEPADQKLSAVGEKQTLSPKHPKPSTVKDCPTLCKQTDNRQTDKSPSQPAANTDRRAEGKKCTEALYAPAEGDKLEAGLSFVHSENRLKGAERPAAGVGKGFPEARGKGPGPQKPPTEADKPNGMKRSPSATGQSSFRSTALPEKSLSCSSSFPETRAGVREASAASSDTSSAKAAGGMLELPAPSNRDHRKAQPAGEGRTHMTKSDSLPSFRVSTLPLESHHPDPNTMGGASHRDRALSVTATVGETKGKDPAPAQPPPARKQNVGRDVTKPSPAPNTDRPISLSNEKDFVVRQRRGKESLRSSPHKKAL",
# KCC2D_HUMAN
"MASTTTCTRFTDEYQLFEELGKGAFSVVRRCMKIPTGQEYAAKIINTKKLSARDHQKLEREARICRLLKHPNIVRLHDSISEEGFHYLVFDLVTGGELFEDIVAREYYSEADASHCIQQILESVNHCHLNGIVHRDLKPENLLLASKSKGAAVKLADFGLAIEVQGDQQAWFGFAGTPGYLSPEVLRKDPYGKPVDMWACGVILYILLVGYPPFWDEDQHRLYQQIKAGAYDFPSPEWDTVTPEAKDLINKMLTINPAKRITASEALKHPWICQRSTVASMMHRQETVDCLKKFNARRKLKGAILTTMLATRNFSAAKSLLKKPDGVKESTESSNTTIEDEDVKARKQEIIKVTEQLIEAINNGDFEAYTKICDPGLTAFEPEALGNLVEGMDFHRFYFENALSKSNKPIHTIILNPHVHLVGDDAACIAYIRLTQYMDGSGMPKTMQSEETRVWHRRDGKWQNVHFHRSGSPTVPIKPPCIPNGKENFSGGTSLWQNI",
# BMP2K_HUMAN
"MKKFSRMPKSEGGSGGGAAGGGAGGAGAGAGCGSGGSSVGVRVFAVGRHQVTLEESLAEGGFSTVFLVRTHGGIRCALKRMYVNNMPDLNVCKREITIMKELSGHKNIVGYLDCAVNSISDNVWEVLILMEYCRAGQVVNQMNKKLQTGFTEPEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLNDGGNYVLCDFGSATNKFLNPQKDGVNVVEEEIKKYTTLSYRAPEMINLYGGKPITTKADIWALGCLLYKLCFFTLPFGESQVAICDGNFTIPDNSRYSRNIHCLIRFMLEPDPEHRPDIFQVSYFAFKFAKKDCPVSNINNSSIPSALPEPMTASEAAARKSQIKARITDTIGPTETSIAPRQRPKANSATTATPSVLTIQSSATPVKVLAPGEFGNHRPKGALRPGNGPEILLGQGPPQQPPQQHRVLQQLQQGDWRLQQLHLQHRHPHQQQQQQQQQQQQQQQQQQQQQQQQQQQHHHHHHHHLLQDAYMQQYQHATQQQQMLQQQFLMHSVYQPQPSASQYPTMMPQYQQAFFQQQMLAQHQPSQQQASPEYLTSPQEFSPALVSYTSSLPAQVGTIMDSSYSANRSVADKEAIANFTNQKNISNPPDMSGWNPFGEDNFSKLTEEELLDREFDLLRSNRLEERASSDKNVDSLSAPHNHPPEDPFGSVPFISHSGSPEKKAEHSSINQENGTANPIKNGKTSPASKDQRTGKKTSVQGQVQKGNDESESDFESDPPSPKSSEEEEQDDEEVLQGEQGDFNDDDTEPENLGHRPLLMDSEDEEEEEKHSSDSDYEQAKAKYSDMSSVYRDRSGSGPTQDLNTILLTSAQLSSDVAVETPKQEFDVFGAVPFFAVRAQQPQQEKNEKNLPQHRFPAAGLEQEEFDVFTKAPFSKKVNVQECHAVGPEAHTIPGYPKSVDVFGSTPFQPFLTSTSKSESNEDLFGLVPFDEITGSQQQKVKQRSLQKLSSRQRRTKQDMSKSNGKRHHGTPTSTKKTLKPTYRTPERARRHKKVGRRDSQSSNEFLTISDSKENISVALTDGKDRGNVLQPEESLLDPFGAKPFHSPDLSWHPPHQGLSDIRADHNTVLPGRPRQNSLHGSFHSADVLKMDDFGAVPFTELVVQSITPHQSQQSQPVELDPFGAAPFPSKQ",
# TSSK1_HUMAN
"MDDAAVLKRRGYLLGINLGEGSYAKVKSAYSERLKFNVAIKIIDRKKAPADFLEKFLPREIEILAMLNHCSIIKTYEIFETSHGKVYIVMELAVQGDLLELIKTRGALHEDEARKKFHQLSLAIKYCHDLDVVHRDLKCDNLLLDKDFNIKLSDFSFSKRCLRDDSGRMALSKTFCGSPAYAAPEVLQGIPYQPKVYDIWSLGVILYIMVCGSMPYDDSNIKKMLRIQKEHRVNFPRSKHLTGECKDLIYHMLQPDVNRRLHIDEILSHCWMQPKARGSPSVAINKEGESSRGTEPLWTPEPGSDKKSATKLEPEGEAQPQAQPETKPEGTAMQMSRQSEILGFPSKPSTMETEEGPPQQPPETRAQ",
# MYLK4_HUMAN
"MLKVKRLEEFNTCYNSNQLEKMAFFQCREEVEKVKCFLEKNSGDQDSRSGHNEAKEVWSNADLTERMPVKSKRTSALAVDIPAPPAPFDHRIVTAKQGAVNSFYTVSKTEILGGGRFGQVHKCEETATGLKLAAKIIKTRGMKDKEEVKNEISVMNQLDHANLIQLYDAFESKNDIVLVMEYVDGGELFDRIIDESYNLTELDTILFMKQICEGIRHMHQMYILHLDLKPENILCVNRDAKQIKIIDFGLARRYKPREKLKVNFGTPEFLAPEVVNYDFVSFPTDMWSVGVIAYMLLSGLSPFLGDNDAETLNNILACRWDLEDEEFQDISEEAKEFISKLLIKEKSWRISASEALKHPWLSDHKLHSRLNAQKKKNRGSDAQDFVTK",
# ADCK2_HUMAN
"MVAPWRVSVRVCLSHLRCFELRQGLSLLRPSECPRDARLCWLLLGTLPKVVSLCGDVGEGAPDVLSRRRVRCSGAAGAGPAESLPRAGPLGGVFLHLRLWLRAGALLVKFFPLLLLYPLTYLAPSVSTLWLHLLLKATETSGPTYIKLGQWASTRRDLFSEAFCAQFSKLHVRVTPHPWTHTERFLRQAFGDDWGSILSFENREPVGSGCVAQVYKAYANTAFLETDSVQRLGRASCLPPFSHTGAVGGLRELFGYLGNGRKPPENLADQSFLERLLLPKADLVGSNAGVSRAQVPGHQPEATNLISVAVKVLHPGLLAQVHMDLLLMKIGSRVLGVLPGIKWLSLPEIVEEFEKLMVQQIDLRYEAQNLEHFQVNFRNVKAVKFPTPLRPFVTREVLVETYEESVPVSSYQQAGIPVDLKRKIARLGINMLLKMIFVDNFVHADLHPGNILVQGANGLSSSQEAQLQQADICDTLVVAVPSSLCPLRLVLLDAGIVAELQAPDLRNFRAVFMAVVMGQGQRVAELILHHARASECRDVEGFKTEMAMLVTQARKNTITLEKLHVSSLLSSVFKLLMTHKVKLESNFASIVFAIMVLEGLGRSLDPKLDILEAARPFLLTGPVCPP",
# DAPK3_HUMAN
"MSTFRQEDVEDHYEMGEELGSGQFAIVRKCRQKGTGKEYAAKFIKKRRLSSSRRGVSREEIEREVNILREIRHPNIITLHDIFENKTDVVLILELVSGGELFDFLAEKESLTEDEATQFLKQILDGVHYLHSKRIAHFDLKPENIMLLDKNVPNPRIKLIDFGIAHKIEAGNEFKNIFGTPEFVAPEIVNYEPLGLEADMWSIGVITYILLSGASPFLGETKQETLTNISAVNYDFDEEYFSNTSELAKDFIRRLLVKDPKRRMTIAQSLEHSWIKAIRRRNVRGEDSGRKPERRRLKTTRLKEYTIKSHSSLPPNNSYADFERFSKVLEEAAAAEEGLRELQRSRRLCHEDVEALAAIYEEKEAWYREESDSLGQDLRRLRQELLKTEALKRQAQEEAKGALLGTSGLKRRFSRLENRYEALAKQVASEMRFVQDLVRALEQEKLQGVECGLR",
# WEE2_HUMAN
"MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEVQDSEAKGTPPWTPLSNVHELDTSSEKDKESPDQILRTPVSHPLKCPETPAQPDSRSKLLPSDSPSTPKTMLSRLVISPTGKLPSRGPKHLKLTPAPLKDEMTSLALVNINPFTPESYKKLFLQSGGKRKIRGDLEEAGPEEGKGGLPAKRCVLRETNMASRYEKEFLEVEKIGVGEFGTVYKCIKRLDGCVYAIKRSMKTFTELSNENSALHEVYAHAVLGHHPHVVRYYSSWAEDDHMIIQNEYCNGGSLQAAISENTKSGNHFEEPKLKDILLQISLGLNYIHNSSMVHLDIKPSNIFICHKMQSESSGVIEEVENEADWFLSANVMYKIGDLGHATSINKPKVEEGDSRFLANEILQEDYRHLPKADIFALGLTIAVAAGAESLPTNGAAWHHIRKGNFPDVPQELSESFSSLLKNMIQPDAEQRPSAAALARNTVLRPSLGKTEELQQQLNLEKFKTATLERELREAQQAQSPQGYTHHGDTGVSGTHTGSRSTKRLVGGKSARSSSFTSGEREPLH",
# RN5A_HUMAN
"MESRDHNNPQEGPTSSSGRRAAVEDNHLLIKAVQNEDVDLVQQLLEGGANVNFQEEEGGWTPLHNAVQMSREDIVELLLRHGADPVLRKKNGATPFILAAIAGSVKLLKLFLSKGADVNECDFYGFTAFMEAAVYGKVKALKFLYKRGANVNLRRKTKEDQERLRKGGATALMDAAEKGHVEVLKILLDEMGADVNACDNMGRNALIHALLSSDDSDVEAITHLLLDHGADVNVRGERGKTPLILAVEKKHLGLVQRLLEQEHIEINDTDSDGKTALLLAVELKLKKIAELLCKRGASTDCGDLVMTARRNYDHSLVKVLLSHGAKEDFHPPAEDWKPQSSHWGAALKDLHRIYRPMIGKLKFFIDEKYKIADTSEGGIYLGFYEKQEVAVKTFCEGSPRAQREVSCLQSSRENSHLVTFYGSESHRGHLFVCVTLCEQTLEACLDVHRGEDVENEEDEFARNVLSSIFKAVQELHLSCGYTHQDLQPQNILIDSKKAAHLADFDKSIKWAGDPQEVKRDLEDLGRLVLYVVKKGSISFEDLKAQSNEEVVQLSPDEETKDLIHRLFHPGEHVRDCLSDLLGHPFFWTWESRYRTLRNVGNESDIKTRKSESEILRLLQPGPSEHSKSFDKWTTKINECVMKKMNKFYEKRGNFYQNTVGDLLKFIRNLGEHIDEEKHKKMKLKIGDPSLYFQKTFPDLVIYVYTKLQNTEYRKHFPQTHSPNKPQCDGAGGASGLASPGC",
# TRPM6_HUMAN
"MKEQPVLERLQSQKSWIKGVFDKRECSTIIPSSKNPHRCTPVCQVCQNLIRCYCGRLIGDHAGIDYSWTISAAKGKESEQWSVEKHTTKSPTDTFGTINFQDGEHTHHAKYIRTSYDTKLDHLLHLMLKEWKMELPKLVISVHGGIQNFTMPSKFKEIFSQGLVKAAETTGAWIITEGINTGVSKHVGDALKSHSSHSLRKIWTVGIPPWGVIENQRDLIGKDVVCLYQTLDNPLSKLTTLNSMHSHFILSDDGTVGKYGNEMKLRRNLEKYLSLQKIHCRSRQGVPVVGLVVEGGPNVILSVWETVKDKDPVVVCEGTGRAADLLAFTHKHLADEGMLRPQVKEEIICMIQNTFNFSLKQSKHLFQILMECMVHRDCITIFDADSEEQQDLDLAILTALLKGTNLSASEQLNLAMAWDRVDIAKKHILIYEQHWKPDALEQAMSDALVMDRVDFVKLLIEYGVNLHRFLTIPRLEELYNTKQGPTNTLLHHLVQDVKQHTLLSGYRITLIDIGLVVEYLIGRAYRSNYTRKHFRALYNNLYRKYKHQRHSSGNRNESAESTLHSQFIRTAQPYKFKEKSIVLHKSRKKSKEQNVSDDPESTGFLYPYNDLLVWAVLMKRQKMAMFFWQHGEEATVKAVIACILYRAMAHEAKESHMVDDASEELKNYSKQFGQLALDLLEKAFKQNERMAMTLLTYELRNWSNSTCLKLAVSGGLRPFVSHTCTQMLLTDMWMGRLKMRKNSWLKIIISIILPPTILTLEFKSKAEMSHVPQSQDFQFMWYYSDQNASSSKESASVKEYDLERGHDEKLDENQHFGLESGHQHLPWTRKVYEFYSAPIVKFWFYTMAYLAFLMLFTYTVLVEMQPQPSVQEWLVSIYIFTNAIEVVREICISEPGKFTQKVKVWISEYWNLTETVAIGLFSAGFVLRWGDPPFHTAGRLIYCIDIIFWFSRLLDFFAVNQHAGPYVTMIAKMTANMFYIVIIMAIVLLSFGVARKAILSPKEPPSWSLARDIVFEPYWMIYGEVYAGEIDVCSSQPSCPPGSFLTPFLQAVYLFVQYIIMVNLLIAFFNNVYLDMESISNNLWKYNRYRYIMTYHEKPWLPPPLILLSHVGLLLRRLCCHRAPHDQEEGDVGLKLYLSKEDLKKLHDFEEQCVEKYFHEKMEDVNCSCEERIRVTSERVTEMYFQLKEMNEKVSFIKDSLLSLDSQVGHLQDLSALTVDTLKVLSAVDTLQEDEALLAKRKHSTCKKLPHSWSNVICAEVLGSMEIAGEKKYQYYSMPSSLLRSLAGGRHPPRVQRGALLEITNSKREATNVRNDQERQETQSSIVVSGVSPNRQAHSKYGQFLLVPSNLKRVPFSAETVLPLSRPSVPDVLATEQDIQTEVLVHLTGQTPVVSDWASVDEPKEKHEPIAHLLDGQDKAEQVLPTLSCTPEPMTMSSPLSQAKIMQTGGGYVNWAFSEGDETGVFSIKKKWQTCLPSTCDSDSSRSEQHQKQAQDSSLSDNSTRSAQSSECSEVGPWLQPNTSFWINPLRRYRPFARSHSFRFHKEEKLMKICKIKNLSGSSEIGQGAWVKAKMLTKDRRLSKKKKNTQGLQVPIITVNACSQSDQLNPEPGENSISEEEYSKNWFTVSKFSHTGVEPYIHQKMKTKEIGQCAIQISDYLKQSQEDLSKNSLWNSRSTNLNRNSLLKSSIGVDKISASLKSPQEPHHHYSAIERNNLMRLSQTIPFTPVQLFAGEEITVYRLEESSPLNLDKSMSSWSQRGRAAMIQVLSREEMDGGLRKAMRVVSTWSEDDILKPGQVFIVKSFLPEVVRTWHKIFQESTVLHLCLREIQQQRAAQKLIYTFNQVKPQTIPYTPRFLEVFLIYCHSANQWLTIEKYMTGEFRKYNNNNGDEITPTNTLEELMLAFSHWTYEYTRGELLVLDLQGVGENLTDPSVIKPEVKQSRGMVFGPANLGEDAIRNFIAKHHCNSCCRKLKLPDLKRNDYSPERINSTFGLEIKIESAEEPPARETGRNSPEDDMQL",
# RIOK2_HUMAN
"MGKVNVAKLRYMSRDDFRVLTAVEMGMKNHEIVPGSLIASIASLKHGGCNKVLRELVKHKLIAWERTKTVQGYRLTNAGYDYLALKTLSSRQVVESVGNQMGVGKESDIYIVANEEGQQFALKLHRLGRTSFRNLKNKRDYHKHRHNVSWLYLSRLSAMKEFAYMKALYERKFPVPKPIDYNRHAVVMELINGYPLCQIHHVEDPASVYDEAMELIVKLANHGLIHGDFNEFNLILDESDHITMIDFPQMVSTSHPNAEWYFDRDVKCIKDFFMKRFSYESELFPTFKDIRREDTLDVEVSASGYTKEMQADDELLHPLGPDDKNIETKEGSEFSFSDGEVAEKAEVYGSENESERNCLEESEGCYCRSSGDPEQIKEDSLSEESADARSFEMTEFNQALEEIKGQVVENNSVTEFSEEKNRTENYNRQDGQRVQGGVPAGSDEYEDECPHLIALSSLNREFRPFRDEENVGAMNQYRTRTLSITSSGSAVSCSTIPPELVKQKVKRQLTKQQKSAVRRRLQKGEANIFTKQRRENMQNIKSSLEAASFWGE",
# ALPK2_HUMAN
"MKDSEGPQRPPLCFLSTLLSQKVPEKSDAVLRCIISGQPKPEVTWYKNGQAIDGSGIISNYEFFENQYIHVLHLSCCTKNDAAVYQISAKNSFGMICCSASVEVECSSENPQLSPNLEDDRDRGWKHETGTHEEERANQIDEKEHPYKEEESISPGTPRSADSSPSKSNHSLSLQSLGNLDISVSSSENPLGVKGTRHTGEAYDPSNTEEIANGLLFLNSSHIYEKQDRCCHKTVHSMASKFTDGDLNNDGPHDEGLRSSQQNPKVQKYISFSLPLSEATAHIYPGDSAVANKQPSPQLSSEDSDSDYELCPEITLTYTEEFSDDDLEYLECSDVMTDYSNAVWQRNLLGTEHVFLLESDDEEMEFGEHCLGGCEHFLSGMGCGSRVSGDAGPMVATAGFCGHHSQPQEVGVRSSRVSKHGPSSPQTGMTLILGPHQDGTSSVTEQGRYKLPTAPEAAENDYPGIQGETRDSHQAREEFASDNLLNMDESVRETEMKLLSGESENSGMSQCWETAADKRVGGKDLWSKRGSRKSARVRQPGMKGNPKKPNANLRESTTEGTLHLCSAKESAEPPLTQSDKRETSHTTAAATGRSSHADARECAISTQAEQEAKTLQTSTDSVSKEGNTNCKGEGMQVNTLFETSQVPDWSDPPQVQVQETVRETISCSQMPAFSEPAGEESPFTGTTTISFSNLGGVHKENASLAQHSEVKPCTCGPQHEEKQDRDGNIPDNFREDLKYEQSISEANDETMSPGVFSRHLPKDARADFREPVAVSVASPEPTDTALTLENVCDEPRDREAVCAMECFEAGDQGTCFDTIDSLVGRPVDKYSPQEICSVDTELAEGQNKVSDLCSSNDKTLEVFFQTQVSETSVSTCKSSKDGNSVMSPLFTSTFTLNISHTASEGATGENLAKVENSTYPLASTVHAGQEQPSPSNSGGLDETQLLSSENNPLVQFKEGGDKSPSPSAADTTATPASYSSIVSFPWEKPTTLTANNECFQATRETEDTSTVTIATEVHPAKYLAVSIPEDKHAGGTEERFPRASHEKVSQFPSQVQLDHILSGATIKSTKELLCRAPSVPGVPHHVLQLPEGEGFCSNSPLQVDNLSGDKSQTVDRADFRSYEENFQERGSETKQGVQQQSLSQQGSLSAPDFQQSLPTTSAAQEERNLVPTAHSPASSREGAGQRSGWGTRVSVVAETAGEEDSQALSNVPSLSDILLEESKEYRPGNWEAGNKLKIITLEASASEIWPPRQLTNSESKASDGGLIIPDKVWAVPDSLKADAVVPELAPSEIAALAHSPEDAESALADSRESHKGEEPTISVHWRSLSSRGFSQPRLLESSVDPVDEKELSVTDSLSAASETGGKENVNNVSQDQEEKQLKMDHTAFFKKFLTCPKILESSVDPIDEISVIEYTRAGKPEPSETTPQGAREGGQSNDGNMGHEAEIQPAILQVPCLQGTILSENRISRSQEGSMKQEAEQIQPEEAKTAIWQVLQPSEGGERIPSGCSIGQIQESSDGSLGEAEQSKKDKAELISPTSPLSSCLPIMTHASLGVDTHNSTGQIHDVPENDIVEPRKRQYVFPVSQKRGTIENERGKPLPSSPDLTRFPCTSSPEGNVTDFLISHKMEEPKIEVLQIGETKPPSSSSSSAKTLAFISGERELEKAPKLLQDPCQKGTLGCAKKSREREKSLEARAGKSPGTLTAVTGSEEVKRKPEAPGSGHLAEGVKKKILSRVAALRLKLEEKENIRKNSAFLKKMPKLETSLSHTEEKQDPKKPSCKREGRAPVLLKKIQAEMFPEHSGNVKLSCQFAEIHEDSTICWTKDSKSIAQVQRSAGDNSTVSFAIVQASPKDQGLYYCCIKNSYGKVTAEFNLTAEVLKQLSSRQDTKGCEEIEFSQLIFKEDFLHDSYFGGRLRGQIATEELHFGEGVHRKAFRSTVMHGLMPVFKPGHACVLKVHNAIAYGTRNNDELIQRNYKLAAQECYVQNTARYYAKIYAAEAQPLEGFGEVPEIIPIFLIHRPENNIPYATVEEELIGEFVKYSIRDGKEINFLRRESEAGQKCCTFQHWVYQKTSGCLLVTDMQGVGMKLTDVGIATLAKGYKGFKGNCSMTFIDQFKALHQCNKYCKMLGLKSLQNNNQKQKQPSIGKSKVQTNSMTIKKAGPETPGEKKT",
# ALPK1_HUMAN
"MNNQKVVAVLLQECKQVLDQLLLEAPDVSEEDKSEDQRCRALLPSELRTLIQEAKEMKWPFVPEKWQYKQAVGPEDKTNLKDVIGAGLQQLLASLRASILARDCAAAAAIVFLVDRFLYGLDVSGKLLQVAKGLHKLQPATPIAPQVVIRQARISVNSGKLLKAEYILSSLISNNGATGTWLYRNESDKVLVQSVCIQIRGQILQKLGMWYEAAELIWASIVGYLALPQPDKKGLSTSLGILADIFVSMSKNDYEKFKNNPQINLSLLKEFDHHLLSAAEACKLAAAFSAYTPLFVLTAVNIRGTCLLSYSSSNDCPPELKNLHLCEAKEAFEIGLLTKRDDEPVTGKQELHSFVKAAFGLTTVHRRLHGETGTVHAASQLCKEAMGKLYNFSTSSRSQDREALSQEVMSVIAQVKEHLQVQSFSNVDDRSYVPESFECRLDKLILHGQGDFQKILDTYSQHHTSVCEVFESDCGNNKNEQKDAKTGVCITALKTEIKNIDTVSTTQEKPHCQRDTGISSSLMGKNVQRELRRGGRRNWTHSDAFRVSLDQDVETETEPSDYSNGEGAVFNKSLSGSQTSSAWSNLSGFSSSASWEEVNYHVDDRSARKEPGKEHLVDTQCSTALSEELENDREGRAMHSLHSQLHDLSLQEPNNDNLEPSQNQPQQQMPLTPFSPHNTPGIFLAPGAGLLEGAPEGIQEVRNMGPRNTSAHSRPSYRSASWSSDSGRPKNMGTHPSVQKEEAFEIIVEFPETNCDVKDRQGKEQGEEISERGAGPTFKASPSWVDPEGETAESTEDAPLDFHRVLHNSLGNISMLPCSSFTPNWPVQNPDSRKSGGPVAEQGIDPDASTVDEEGQLLDSMDVPCTNGHGSHRLCILRQPPGQRAETPNSSVSGNILFPVLSEDCTTTEEGNQPGNMLNCSQNSSSSSVWWLKSPAFSSGSSEGDSPWSYLNSSGSSWVSLPGKMRKEILEARTLQPDDFEKLLAGVRHDWLFQRLENTGVFKPSQLHRAHSALLLKYSKKSELWTAQETIVYLGDYLTVKKKGRQRNAFWVHHLHQEEILGRYVGKDYKEQKGLWHHFTDVERQMTAQHYVTEFNKRLYEQNIPTQIFYIPSTILLILEDKTIKGCISVEPYILGEFVKLSNNTKVVKTEYKATEYGLAYGHFSYEFSNHRDVVVDLQGWVTGNGKGLIYLTDPQIHSVDQKVFTTNFGKRGIFYFFNNQHVECNEICHRLSLTRPSMEKPCT",
# BRD2_HUMAN
"MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESPTMASVPALQLTPANPPPPEVSNPKKPGRVTNQLQYLHKVVMKALWKHQFAWPFRQPVDAVKLGLPDYHKIIKQPMDMGTIKRRLENNYYWAASECMQDFNTMFTNCYIYNKPTDDIVLMAQTLEKIFLQKVASMPQEEQELVVTIPKNSHKKGAKLAALQGSVTSAHQVPAVSSVSHTALYTPPPEIPTTVLNIPHPSVISSPLLKSLHSAGPPLLAVTAAPPAQPLAKKKGVKRKADTTTPTPTAILAPGSPASPPGSLEPKAARLPPMRRESGRPIKPPRKDLPDSQQQHQSSKKGKLSEQLKHCNGILKELLSKKHAAYAWPFYKPVDASALGLHDYHDIIKHPMDLSTVKRKMENRDYRDAQEFAADVRLMFSNCYKYNPPDHDVVAMARKLQDVFEFRYAKMPDEPLEPGPLPVSTAMPPGLAKSSSESSSEESSSESSSEEEEEEDEEDEEEEESESSDSEEERAHRLAELQEQLRAVHEQLAALSQGPISKPKRKREKKEKKKKRKAEKHRGRAGADEDDKGPRAPRPPQPKKSKKASGSGGGSAALGPSGFGPSGGSGTKLPKKATKTAPPALPTGYDSEEEEESRPMSYDEKRQLSLDINKLPGEKLGRVVHIIQAREPSLRDSNPEEIEIDFETLKPSTLRELERYVLSCLRKKPRKPYTIKKPVGKTKEELALEKKRELEKRLQDVSGQLNSTKKPPKKANEKTESSSAQQVAVSRLSASSSSSDSSSSSSSSSSSDTSDSDSG",
# BRD3_HUMAN
"MSTATTVAPAGIPATPGPVNPPPPEVSNPSKPGRKTNQLQYMQNVVVKTLWKHQFAWPFYQPVDAIKLNLPDYHKIIKNPMDMGTIKKRLENNYYWSASECMQDFNTMFTNCYIYNKPTDDIVLMAQALEKIFLQKVAQMPQEEVELLPPAPKGKGRKPAAGAQSAGTQQVAAVSSVSPATPFQSVPPTVSQTPVIAATPVPTITANVTSVPVPPAAAPPPPATPIVPVVPPTPPVVKKKGVKRKADTTTPTTSAITASRSESPPPLSDPKQAKVVARRESGGRPIKPPKKDLEDGEVPQHAGKKGKLSEHLRYCDSILREMLSKKHAAYAWPFYKPVDAEALELHDYHDIIKHPMDLSTVKRKMDGREYPDAQGFAADVRLMFSNCYKYNPPDHEVVAMARKLQDVFEMRFAKMPDEPVEAPALPAPAAPMVSKGAESSRSSEESSSDSGSSDSEEERATRLAELQEQLKAVHEQLAALSQAPVNKPKKKKEKKEKEKKKKDKEKEKEKHKVKAEEEKKAKVAPPAKQAQQKKAPAKKANSTTTAGRQLKKGGKQASASYDSEEEEEGLPMSYDEKRQLSLDINRLPGEKLGRVVHIIQSREPSLRDSNPDEIEIDFETLKPTTLRELERYVKSCLQKKQRKPFSASGKKQAAKSKEELAQEKKKELEKRLQDVSGQLSSSKKPARKEKPGSAPSGGPSRLSSSSSSESGSSSSSGSSSDSSDSE",
# BRD4_HUMAN
"MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPVDAVKLNLPDYYKIIKTPMDMGTIKKRLENNYYWNAQECIQDFNTMFTNCYIYNKPGDDIVLMAEALEKLFLQKINELPTEETEIMIVQAKGRGRGRKETGTAKPGVSTVPNTTQASTPPQTQTPQPNPPPVQATPHPFPAVTPDLIVQTPVMTVVPPQPLQTPPPVPPQPQPPPAPAPQPVQSHPPIIAATPQPVKTKKGVKRKADTTTPTTIDPIHEPPSLPPEPKTTKLGQRRESSRPVKPPKKDVPDSQQHPAPEKSSKVSEQLKCCSGILKEMFAKKHAAYAWPFYKPVDVEALGLHDYCDIIKHPMDMSTIKSKLEAREYRDAQEFGADVRLMFSNCYKYNPPDHEVVAMARKLQDVFEMRFAKMPDEPEEPVVAVSSPAVPPPTKVVAPPSSSDSSSDSSSDSDSSTDDSEEERAQRLAELQEQLKAVHEQLAALSQPQQNKPKKKEKDKKEKKKEKHKRKEEVEENKKSKAKEPPPKKTKKNNSSNSNVSKKEPAPMKSKPPPTYESEEEDKCKPMSYEEKRQLSLDINKLPGEKLGRVVHIIQSREPSLKNSNPDEIEIDFETLKPSTLRELERYVTSCLRKKRKPQAEKVDVIAGSSKMKGFSSSESESSSESSSSDSEDSETEMAPKSKKKGHPGREQKKHHHHHHQQMQQAPAPVPQQPPPPPQQPPPPPPPQQQQQPPPPPPPPSMPQQAAPAMKSSPPPFIATQVPVLEPQLPGSVFDPIGHFTQPILHLPQPELPPHLPQPPEHSTPPHLNQHAVVSPPALHNALPQQPSRPSNRAAALPPKPARPPAVSPALTQTPLLPQPPMAQPPQVLLEDEEPPAPPLTSMQMQLYLQQLQKVQPPTPLLPSVKVQSQPPPPLPPPPHPSVQQQLQQQPPPPPPPQPQPPPQQQHQPPPRPVHLQPMQFSTHIQQPPPPQGQQPPHPPPGQQPPPPQPAKPQQVIQHHHSPRHHKSDPYSTGHLREAPSPLMIHSPQMSQFQSLTHQSPPQQNVQPKKQELRAASVVQPQPLVVVKEEKIHSPIIRSEPFSPSLRPEPPKHPESIKAPVHLPQRPEMKPVDVGRPVIRPPEQNAPPPGAPDKDKQKQEPKTPVAPKKDLKIKNMGSWASLVQKHPTTPSSTAKSSSDSFEQFRRAAREKEEREKALKAQAEHAEKEKERLRQERMRSREDEDALEQARRAHEEARRRQEQQQQQRQEQQQQQQQQAAAVAAAATPQAQSSQPQSMLDQQRELARKREQERRRREAMAATIDMNFQSDLLSIFEENLF",
# BRDT_HUMAN
"MSLPSRQTAIIVNPPPPEYINTKKNGRLTNQLQYLQKVVLKDLWKHSFSWPFQRPVDAVKLQLPDYYTIIKNPMDLNTIKKRLENKYYAKASECIEDFNTMFSNCYLYNKPGDDIVLMAQALEKLFMQKLSQMPQEEQVVGVKERIKKGTQQNIAVSSAKEKSSPSATEKVFKQQEIPSVFPKTSISPLNVVQGASVNSSSQTAAQVTKGVKRKADTTTPATSAVKASSEFSPTFTEKSVALPPIKENMPKNVLPDSQQQYNVVKTVKVTEQLRHCSEILKEMLAKKHFSYAWPFYNPVDVNALGLHNYYDVVKNPMDLGTIKEKMDNQEYKDAYKFAADVRLMFMNCYKYNPPDHEVVTMARMLQDVFETHFSKIPIEPVESMPLCYIKTDITETTGRENTNEASSEGNSSDDSEDERVKRLAKLQEQLKAVHQQLQVLSQVPFRKLNKKKEKSKKEKKKEKVNNSNENPRKMCEQMRLKEKSKRNQPKKRKQQFIGLKSEDEDNAKPMNYDEKRQLSLNINKLPGDKLGRVVHIIQSREPSLSNSNPDEIEIDFETLKASTLRELEKYVSACLRKRPLKPPAKKIMMSKEELHSQKKQELEKRLLDVNNQLNSRKRQTKSDKTQPSKAVENVSRLSESSSSSSSSSESESSSSDLSSSDSSDSESEMFPKFTEVKPNDSPSKENVKKMKNECIPPEGRTGVTQIGYCVQDTTSANTTLVHQTTPSHVMPPNHHQLAFNYQELEHLQTVKNISPLQILPPSGDSEQLSNGITVMHPSGDSDTTMLESECQAPVQKDIKIKNADSWKSLGKPVKPSGVMKSSDELFNQFRKAAIEKEVKARTQELIRKHLEQNTKELKASQENQRDLGNGLTVESFSNKIQNKCSGEEQKEHQQSSEAQDKSKLWLLKDRDLARQKEQERRRREAMVGTIDMTLQSDIMTMFENNFD",
# ALPK3_HUMAN
"MGSRRAPSRGWGAGGRSGAGGDGEDDGPVWIPSPASRSYLLSVRPETSLSSNRLSHPSSGRSTFCSIIAQLTEETQPLFETTLKSRSVSEDSDVRFTCIVTGYPEPEVTWYKDDTELDRYCGLPKYEITHQGNRHTLQLYRCREEDAAIYQASAQNSKGIVSCSGVLEVGTMTEYKIHQRWFAKLKRKAAAKLREIEQSWKHEKAVPGEVDTLRKLSPDRFQRKRRLSGAQAPGPSVPTREPEGGTLAAWQEGETETAQHSGLGLINSFASGEVTTNGEAAPENGEDGEHGLLTYICDAMELGPQRALKEESGAKKKKKDEESKQGLRKPELEKAAQSRRSSENCIPSSDEPDSCGTQGPVGVEQVQTQPRGRAARGPGSSGTDSTRKPASAVGTPDKAQKAPGPGPGQEVYFSLKDMYLENTQAVRPLGEEGPQTLSVRAPGESPKGKAPLRARSEGVPGAPGQPTHSLTPQPTRPFNRKRFAPPKPKGEATTDSKPISSLSQAPECGAQSLGKAPPQASVQVPTPPARRRHGTRDSTLQGQAGHRTPGEVLECQTTTAPTMSASSSSDVASIGVSTSGSQGIIEPMDMETQEDGRTSANQRTGSKKNVQADGKIQVDGRTRGDGTQTAQRTRADRKTQVDAGTQESKRPQSDRSAQKGMMTQGRAETQLETTQAGEKIQEDRKAQADKGTQEDRRMQGEKGMQGEKGTQSEGSAPTAMEGQSEQEVATSLGPPSRTPKLPPTAGPRAPLNIECFVQTPEGSCFPKKPGCLPRSEEAVVTASRNHEQTVLGPLSGNLMLPAQPPHEGSVEQVGGERCRGPQSSGPVEAKQEDSPFQCPKEERPGGVPCMDQGGCPLAGLSQEVPTMPSLPGTGLTASPKAGPCSTPTSQHGSTATFLPSEDQVLMSSAPTLHLGLGTPTQSHPPETMATSSEGACAQVPDVEGRTPGPRSCDPGLIDSLKNYLLLLLKLSSTETSGAGGESQVGAATGGLVPSATLTPTVEVAGLSPRTSRRILERVENNHLVQSAQTLLLSPCTSRRLTGLLDREVQAGRQALAAARGSWGPGPSSLTVPAIVVDEEDPGLASEGASEGEGEVSPEGPGLLGASQESSMAGRLGEAGGQAAPGQGPSAESIAQEPSQEEKFPGEALTGLPAATPEELALGARRKRFLPKVRAAGDGEATTPEERESPTVSPRGPRKSLVPGSPGTPGRERRSPTQGRKASMLEVPRAEEELAAGDLGPSPKAGGLDTEVALDEGKQETLAKPRKAKDLLKAPQVIRKIRVEQFPDASGSLKLWCQFFNILSDSVLTWAKDQRPVGEVGRSAGDEGPAALAIVQASPVDCGVYRCTIHNEHGSASTDFCLSPEVLSGFISREEGEVGEEIEMTPMVFAKGLADSGCWGDKLFGRLVSEELRGGGYGCGLRKASQAKVIYGLEPIFESGRTCIIKVSSLLVFGPSSETSLVGRNYDVTIQGCKIQNMSREYCKIFAAEARAAPGFGEVPEIIPLYLIYRPANNIPYATLEEDLGKPLESYCSREWGCAEAPTASGSSEAMQKCQTFQHWLYQWTNGSFLVTDLAGVDWKMTDVQIATKLRGYQGLKESCFPALLDRFASSHQCNAYCELLGLTPLKGPEAAHPQAKAKGSKSPSAGRKGSQLSPQPQKKGLPSPQGTRKSAPSSKATPQASEPVTTQLLGQPPTQEEGSKAQGMR",
# ADCK5_HUMAN
"MWRPVQLCHFHSALLHSRQKPWPSPAVFFRRNVRGLPPRFSSPTPLWRKVLSTAVVGAPLLLGARYVMAEAREKRRMRLVVDGMGRFGRSLKVGLQISLDYWWCTNVVLRGVEENSPGYLEVMSACHQRAADALVAGAISNGGLYVKLGQGLCSFNHLLPPEYTRTLRVLEDRALKRGFQEVDELFLEDFQALPHELFQEFDYQPIAAASLAQVHRAKLHDGTSVAVKVQYIDLRDRFDGDIHTLELLLRLVEVMHPSFGFSWVLQDLKGTLAQELDFENEGRNAERCARELAHFPYVVVPRVHWDKSSKRVLTADFCAGCKVNDVEAIRSQGLAVHDIAEKLIKAFAEQIFYTGFIHSDPHPGNVLVRKGPDGKAELVLLDHGLYQFLEEKDRAALCQLWRAIILRDDAAMRAHAAALGVQDYLLFAEMLMQRPVRLGQLWGSHLLSREEAAYMVDMARERFEAVMAVLRELPRPMLLVLRNINTVRAINVALGAPVDRYFLMAKRAVRGWSRLAGATYRGVYGTSLLRHAKVVWEMLKFEVALRLETLAMRLTALLARALVHLSLVPPAEELYQYLET",
# TIF1A_HUMAN
"MEVAVEKAVAAAAAASAAASGGPSAAPSGENEAESRQGPDSERGGEAARLNLLDTCAVCHQNIQSRAPKLLPCLHSFCQRCLPAPQRYLMLPAPMLGSAETPPPVPAPGSPVSGSSPFATQVGVIRCPVCSQECAERHIIDNFFVKDTTEVPSSTVEKSNQVCTSCEDNAEANGFCVECVEWLCKTCIRAHQRVKFTKDHTVRQKEEVSPEAVGVTSQRPVFCPFHKKEQLKLYCETCDKLTCRDCQLLEHKEHRYQFIEEAFQNQKVIIDTLITKLMEKTKYIKFTGNQIQNRIIEVNQNQKQVEQDIKVAIFTLMVEINKKGKALLHQLESLAKDHRMKLMQQQQEVAGLSKQLEHVMHFSKWAVSSGSSTALLYSKRLITYRLRHLLRARCDASPVTNNTIQFHCDPSFWAQNIINLGSLVIEDKESQPQMPKQNPVVEQNSQPPSGLSSNQLSKFPTQISLAQLRLQHMQQQVMAQRQQVQRRPAPVGLPNPRMQGPIQQPSISHQQPPPRLINFQNHSPKPNGPVLPPHPQQLRYPPNQNIPRQAIKPNPLQMAFLAQQAIKQWQISSGQGTPSTTNSTSSTPSSPTITSAAGYDGKAFGSPMIDLSSPVGGSYNLPSLPDIDCSSTIMLDNIVRKDTNIDHGQPRPPSNRTVQSPNSSVPSPGLAGPVTMTSVHPPIRSPSASSVGSRGSSGSSSKPAGADSTHKVPVVMLEPIRIKQENSGPPENYDFPVVIVKQESDEESRPQNANYPRSILTSLLLNSSQSSTSEETVLRSDAPDSTGDQPGLHQDNSSNGKSEWLDPSQKSPLHVGETRKEDDPNEDWCAVCQNGGELLCCEKCPKVFHLSCHVPTLTNFPSGEWICTFCRDLSKPEVEYDCDAPSHNSEKKKTEGLVKLTPIDKRKCERLLLFLYCHEMSLAFQDPVPLTVPDYYKIIKNPMDLSTIKKRLQEDYSMYSKPEDFVADFRLIFQNCAEFNEPDSEVANAGIKLENYFEELLKNLYPEKRFPKPEFRNESEDNKFSDDSDDDFVQPRKKRLKSIEERQLLK",
# TIF1B_HUMAN
"MAASAAAASAAAASAASGSPGPGEGSAGGEKRSTAPSAAASASASAAASSPAGGGAEALELLEHCGVCRERLRPEREPRLLPCLHSACSACLGPAAPAAANSSGDGGAAGDGTVVDCPVCKQQCFSKDIVENYFMRDSGSKAATDAQDANQCCTSCEDNAPATSYCVECSEPLCETCVEAHQRVKYTKDHTVRSTGPAKSRDGERTVYCNVHKHEPLVLFCESCDTLTCRDCQLNAHKDHQYQFLEDAVRNQRKLLASLVKRLGDKHATLQKSTKEVRSSIRQVSDVQKRVQVDVKMAILQIMKELNKRGRVLVNDAQKVTEGQQERLERQHWTMTKIQKHQEHILRFASWALESDNNTALLLSKKLIYFQLHRALKMIVDPVEPHGEMKFQWDLNAWTKSAEAFGKIVAERPGTNSTGPAPMAPPRAPGPLSKQGSGSSQPMEVQEGYGFGSGDDPYSSAEPHVSGVKRSRSGEGEVSGLMRKVPRVSLERLDLDLTADSQPPVFKVFPGSTTEDYNLIVIERGAAAAATGQPGTAPAGTPGAPPLAGMAIVKEEETEAAIGAPPTATEGPETKPVLMALAEGPGAEGPRLASPSGSTSSGLEVVAPEGTSAPGGGPGTLDDSATICRVCQKPGDLVMCNQCEFCFHLDCHLPALQDVPGEEWSCSLCHVLPDLKEEDGSLSLDGADSTGVVAKLSPANQRKCERVLLALFCHEPCRPLHQLATDSTFSLDQPGGTLDLTLIRARLQEKLSPPYSSPQEFAQDVGRMFKQFNKLTEDKADVQSIIGLQRFFETRMNEAFGDTKFSAVLVEPPPMSLPGAGLSSQELSGGPGDGP",
# TRI33_HUMAN
"MAENKGGGEAESGGGGSGSAPVTAGAAGPAAQEAEPPLTAVLVEEEEEEGGRAGAEGGAAGPDDGGVAAASSGSAQAASSPAASVGTGVAGGAVSTPAPAPASAPAPGPSAGPPPGPPASLLDTCAVCQQSLQSRREAEPKLLPCLHSFCLRCLPEPERQLSVPIPGGSNGDIQQVGVIRCPVCRQECRQIDLVDNYFVKDTSEAPSSSDEKSEQVCTSCEDNASAVGFCVECGEWLCKTCIEAHQRVKFTKDHLIRKKEDVSESVGASGQRPVFCPVHKQEQLKLFCETCDRLTCRDCQLLEHKEHRYQFLEEAFQNQKGAIENLLAKLLEKKNYVHFAATQVQNRIKEVNETNKRVEQEIKVAIFTLINEINKKGKSLLQQLENVTKERQMKLLQQQNDITGLSRQVKHVMNFTNWAIASGSSTALLYSKRLITFQLRHILKARCDPVPAANGAIRFHCDPTFWAKNVVNLGNLVIESKPAPGYTPNVVVGQVPPGTNHISKTPGQINLAQLRLQHMQQQVYAQKHQQLQQMRMQQPPAPVPTTTTTTQQHPRQAAPQMLQQQPPRLISVQTMQRGNMNCGAFQAHQMRLAQNAARIPGIPRHSGPQYSMMQPHLQRQHSNPGHAGPFPVVSVHNTTINPTSPTTATMANANRGPTSPSVTAIELIPSVTNPENLPSLPDIPPIQLEDAGSSSLDNLLSRYISGSHLPPQPTSTMNPSPGPSALSPGSSGLSNSHTPVRPPSTSSTGSRGSCGSSGRTAEKTSLSFKSDQVKVKQEPGTEDEICSFSGGVKQEKTEDGRRSACMLSSPESSLTPPLSTNLHLESELDALASLENHVKIEPADMNESCKQSGLSSLVNGKSPIRSLMHRSARIGGDGNNKDDDPNEDWCAVCQNGGDLLCCEKCPKVFHLTCHVPTLLSFPSGDWICTFCRDIGKPEVEYDCDNLQHSKKGKTAQGLSPVDQRKCERLLLYLYCHELSIEFQEPVPASIPNYYKIIKKPMDLSTVKKKLQKKHSQHYQIPDDFVADVRLIFKNCERFNEMMKVVQVYADTQEINLKADSEVAQAGKAVALYFEDKLTEIYSDRTFAPLPEFEQEEDDGEVTEDSDEDFIQPRRKRLKSDERPVHIK",
# PAN3_HUMAN
"MNSGGGLPPPSAAASPSSSSLAAAVAVVAPPGVGGVPGGAAVGVKLKYCRYYAKDKTCFYGEECQFLHEDPAAGAAPGLGLHSNSVPLALAGAPVAGFPPGAVAGGGAGPPPGPKKPDLGDPGTGAAAGGGGSSGGLDGPRLAIPGMDGGALTDTSLTDSYFSTSFIGVNGFGSPVETKYPLMQRMTNSSSSPSLLNDSAKPYSAHDPLTSPASSLFNDFGALNISQRRKPRKYRLGMLEERLVPMGSKARKAKNPIGCLADRCKSGVPINMVWWNRVTENNLQTPNPTASEFIPKGGSTSRLSNVSQSNMSAFSQVFSHPSMGSPATAGLAPGMSLSAGSSPLHSPKITPHTSPAPRRRSHTPNPASYMVPSSASTSVNNPVSQTPSSGQVIQKETVGGTTYFYTDTTPAPLTGMVFPNYHIYPPTAPHVAYMQPKANAPSFFMADELRQELINRHLITMAQIDQADMPAVPTEVDSYHSLFPLEPLPPPNRIQKSSNFGYITSCYKAVNSKDDLPYCLRRIHGFRLVNTKCMVLVDMWKKIQHSNIVTLREVFTTKAFAEPSLVFAYDFHAGGETMMSRHFNDPNADAYFTKRKWGQHEGPLPRQHAGLLPESLIWAYIVQLSSALRTIHTAGLACRVMDPTKILITGKTRLRVNCVGVFDVLTFDNSQNNNPLALMAQYQQADLISLGKVVLALACNSLAGIQRENLQKAMELVTINYSSDLKNLILYLLTDQNRMRSVNDIMPMIGARFYTQLDAAQMRNDVIEEDLAKEVQNGRLFRLLAKLGTINERPEFQKDPTWSETGDRYLLKLFRDHLFHQVTEAGAPWIDLSHIISCLNKLDAGVPEKISLISRDEKSVLVVTYSDLKRCFENTFQELIAAANGQL",
# FASTK_HUMAN
"MRRPRGEPGPRAPRPTEGATCAGPGESWSPSPNSMLRVLLSAQTSPARLSGLLLIPPVQPCCLGPSKWGDRPVGGGPSAGPVQGLQRLLEQAKSPGELLRWLGQNPSKVRAHHYSVALRRLGQLLGSRPRPPPVEQVTLQDLSQLIIRNCPSFDIHTIHVCLHLAVLLGFPSDGPLVCALEQERRLRLPPKPPPPLQPLLRGGQGLEAALSCPRFLRYPRQHLISSLAEARPEELTPHVMVLLAQHLARHRLREPQLLEAIAHFLVVQETQLSSKVVQKLVLPFGRLNYLPLEQQFMPCLERILAREAGVAPLATVNILMSLCQLRCLPFRALHFVFSPGFINYISGTPHALIVRRYLSLLDTAVELELPGYRGPRLPRRQQVPIFPQPLITDRARCKYSHKDIVAEGLRQLLGEEKYRQDLTVPPGYCTDFLLCASSSGAVLPVRTQDPFLPYPPRSCPQGQAASSATTRDPAQRVVLVLRERWHFCRDGRVLLGSRALRERHLGLMGYQLLPLPFEELESQRGLPQLKSYLRQKLQALGLRWGPEGG",
# WHR1_HUMAN
"MSWKRHHLIPETFGVKRRRKRGPVESDPLRGEPGSARAAVSELMQLFPRGLFEDALPPIVLRSQVYSLVPDRTVADRQLKELQEQGEIRIVQLGFDLDAHGIIFTEDYRTRVLKACDGRPYAGAVQKFLASVLPACGDLSFQQDQMTQTFGFRDSEITHLVNAGVLTVRDAGSWWLAVPGAGRFIKYFVKGRQAVLSMVRKAKYRELLLSELLGRRAPVVVRLGLTYHVHDLIGAQLVDCISTTSGTLLRLPET",
# TAF1_HUMAN
"MGPGCDLLLRTAATITAAAIMSDTDSDEDSAGGGPFSLAGFLFGNINGAGQLEGESVLDDECKKHLAGLGALGLGSLITELTANEELTGTDGALVNDEGWVRSTEDAVDYSDINEVAEDESRRYQQTMGSLQPLCHSDYDEDDYDADCEDIDCKLMPPPPPPPGPMKKDKDQDSITGVSENGEGIILPSIIAPSSLASEKVDFSSSSDSESEMGPQEATQAESEDGKLTLPLAGIMQHDATKLLPSVTELFPEFRPGKVLRFLRLFGPGKNVPSVWRSARRKRKKKHRELIQEEQIQEVECSVESEVSQKSLWNYDYAPPPPPEQCLSDDEITMMAPVESKFSQSTGDIDKVTDTKPRVAEWRYGPARLWYDMLGVPEDGSGFDYGFKLRKTEHEPVIKSRMIEEFRKLEENNGTDLLADENFLMVTQLHWEDDIIWDGEDVKHKGTKPQRASLAGWLPSSMTRNAMAYNVQQGFAATLDDDKPWYSIFPIDNEDLVYGRWEDNIIWDAQAMPRLLEPPVLTLDPNDENLILEIPDEKEEATSNSPSKESKKESSLKKSRILLGKTGVIKEEPQQNMSQPEVKDPWNLSNDEYYYPKQQGLRGTFGGNIIQHSIPAVELRQPFFPTHMGPIKLRQFHRPPLKKYSFGALSQPGPHSVQPLLKHIKKKAKMREQERQASGGGEMFFMRTPQDLTGKDGDLILAEYSEENGPLMMQVGMATKIKNYYKRKPGKDPGAPDCKYGETVYCHTSPFLGSLHPGQLLQAFENNLFRAPIYLHKMPETDFLIIRTRQGYYIRELVDIFVVGQQCPLFEVPGPNSKRANTHIRDFLQVFIYRLFWKSKDRPRRIRMEDIKKAFPSHSESSIRKRLKLCADFKRTGMDSNWWVLKSDFRLPTEEEIRAMVSPEQCCAYYSMIAAEQRLKDAGYGEKSFFAPEEENEEDFQMKIDDEVRTAPWNTTRAFIAAMKGKCLLEVTGVADPTGCGEGFSYVKIPNKPTQQKDDKEPQPVKKTVTGTDADLRRLSLKNAKQLLRKFGVPEEEIKKLSRWEVIDVVRTMSTEQARSGEGPMSKFARGSRFSVAEHQERYKEECQRIFDLQNKVLSSTEVLSTDTDSSSAEDSDFEEMGKNIENMLQNKKTSSQLSREREEQERKELQRMLLAAGSAASGNNHRDDDTASVTSLNSSATGRCLKIYRTFRDEEGKEYVRCETVRKPAVIDAYVRIRTTKDEEFIRKFALFDEQHREEMRKERRRIQEQLRRLKRNQEKEKLKGPPEKKPKKMKERPDLKLKCGACGAIGHMRTNKFCPLYYQTNAPPSNPVAMTEEQEEELEKTVIHNDNEELIKVEGTKIVLGKQLIESADEVRRKSLVLKFPKQQLPPKKKRRVGTTVHCDYLNRPHKSIHRRRTDPMVTLSSILESIINDMRDLPNTYPFHTPVNAKVVKDYYKIITRPMDLQTLRENVRKRLYPSREEFREHLELIVKNSATYNGPKHSLTQISQSMLDLCDEKLKEKEDKLARLEKAINPLLDDDDQVAFSFILDNIVTQKMMAVPDSWPFHHPVNKKFVPDYYKVIVNPMDLETIRKNISKHKYQSRESFLDDVNLILANSVKYNGPESQYTKTAQEIVNVCYQTLTEYDEHLTQLEKDICTAKEAALEEAELESLDPMTPGPYTPQPPDLYDTNTSLSMSRDASVFQDESNMSVLDIPSATPEKQVTQEGEDGDGDLADEEEGTVQQPQASVLYEDLLMSEGEDDEEDAGSDEEGDNPFSAIQLSESGSDSDVGSGGIRPKQPRMLQENTRMDMENEESMMSYEGDGGEASHGLEDSNISYGSYEEPDPKSNTQDTSFSSIGGYEVSEEEEDEEEEEQRSGPSVLSQVHLSEDEEDSEDFHSIAGDSDLDSDE",
# TAF1L_HUMAN
"MRPGCDLLLRAAATVTAAIMSDSDSEEDSSGGGPFTLAGILFGNISGAGQLEGESVLDDECKKHLAGLGALGLGSLITELTANEELTGTGGALVNDEGWIRSTEDAVDYSDINEVAEDESQRHQQTMGSLQPLYHSDYDEDDYDADCEDIDCKLMPPPPPPPGPMKKDKDQDAITCVSESGEDIILPSIIAPSFLASEKVDFSSYSDSESEMGPQEATQAESEDGKLTLPLAGIMQHDATKLLPSVTELFPEFRPGKVLRFLHLFGPGKNVPSVWRSARRKRKKHRELIQEEQIQEVECSVESEVSQKSLWNYDYAPPPPPEQCLADDEITMMVPVESKFSQSTGDVDKVTDTKPRVAEWRYGPARLWYDMLGVSEDGSGFDYGFKLRKTQHEPVIKSRMMEEFRKLEESNGTDLLADENFLMVTQLHWEDSIIWDGEDIKHKGTKPQGASLAGWLPSIKTRNVMAYNVQQGFAPTLDDDKPWYSIFPIDNEDLVYGRWEDNIIWDAQAMPRLLEPPVLALDPNDENLILEIPDEKEEATSNSPSKESKKESSLKKSRILLGKTGVIREEPQQNMSQPEVKDPWNLSNDEYYFPKQQGLRGTFGGNIIQHSIPAMELWQPFFPTHMGPIKIRQFHRPPLKKYSFGALSQPGPHSVQPLLKHIKKKAKMREQERQASGGGELFFMRTPQDLTGKDGDLILAEYSEENGPLMMQVGMATKIKNYYKRKPGKDPGAPDCKYGETVYCHTSPFLGSLHPGQLLQALENNLFRAPVYLHKMPETDFLIIRTRQGYYIRELVDIFVVGQQCPLFEVPGPNSRRANMHIRDFLQVFIYRLFWKSKDRPRRIRMEDIKKAFPSHSESSIRKRLKLCADFKRTGMDSNWWVLKSDFRLPTEEEIRAKVSPEQCCAYYSMIAAKQRLKDAGYGEKSFFAPEEENEEDFQMKIDDEVHAAPWNTTRAFIAAMKGKCLLEVTGVADPTGCGEGFSYVKIPNKPTQQKDDKEPQAVKKTVTGTDADLRRLSLKNAKQLLRKFGVPEEEIKKLSRWEVIDVVRTMSTEQAHSGEGPMSKFARGSRFSVAEHQERYKEECQRIFDLQNKVLSSTEVLSTDTDSISAEDSDFEEMGKNIENMLQNKKTSSQLSREWEEQERKELRRMLLVAGSAASGNNHRDDVTASMTSLKSSATGHCLKIYRTFRDEEGKEYVRCETVRKPAVIDAYVRIRTTKDEKFIQKFALFDEKHREEMRKERRRIQEQLRRLKRNQEKEKLKGPPEKKPKKMKERPDLKLKCGACGAIGHMRTNKFCPLYYQTNVPPSKPVAMTEEQEEELEKTVIHNDNEELIKVEGTKIVFGKQLIENVHEVRRKSLVLKFPKQQLPPKKKRRVGTTVHCDYLNIPHKSIHRRRTDPMVTLSSILESIINDMRDLPNTHPFHTPVNAKVVKDYYKIITRPMDLQTLRENVRKCLYPSREEFREHLELIVKNSATYNGPKHSLTQISQSMLDLCDEKLKEKEDKLARLEKAINPLLDDDDQVAFSFILDNIVTQKMMAVPDSWPFHHPVNKKFVPDYYKMIVNPVDLETIRKNISKHKYQSRESFLDDVNLILANSVKYNGPESQYTKTAQEIVNICYQTITEYDEHLTQLEKDICTAKEAALEEAELESLDPMTPGPYTSQPPDMYDTNTSLSTSRDASVFQDESNLSVLDISTATPEKQMCQGQGRLGEEDSDVDVEGYDDEEEDGKPKPPAPEGGDGDLADEEEGTVQQPEASVLYEDLLISEGEDDEEDAGSDEEGDNPFSAIQLSESGSDSDVGYGGIRPKQPFMLQHASGEHKDGHGK",
# BCR_HUMAN
"MVDPVGFAEAWKAQFPDSEPPRMELRSVGDIEQELERCKASIRRLEQEVNQERFRMIYLQTLLAKEKKSYDRQRWGFRRAAQAPDGASEPRASASRPQPAPADGADPPPAEEPEARPDGEGSPGKARPGTARRPGAAASGERDDRGPPASVAALRSNFERIRKGHGQPGADAEKPFYVNVEFHHERGLVKVNDKEVSDRISSLGSQAMQMERKKSQHGAGSSVGDASRPPYRGRSSESSCGVDGDYEDAELNPRFLKDNLIDANGGSRPPWPPLEYQPYQSIYVGGMMEGEGKGPLLRSQSTSEQEKRLTWPRRSYSPRSFEDCGGGYTPDCSSNENLTSSEEDFSSGQSSRVSPSPTTYRMFRDKSRSPSQNSQQSFDSSSPPTPQCHKRHRHCPVVVSEATIVGVRKTGQIWPNDGEGAFHGDADGSFGTPPGYGCAADRAEEQRRHQDGLPYIDDSPSSSPHLSSKGRGSRDALVSGALESTKASELDLEKGLEMRKWVLSGILASEETYLSHLEALLLPMKPLKAAATTSQPVLTSQQIETIFFKVPELYEIHKEFYDGLFPRVQQWSHQQRVGDLFQKLASQLGVYRAFVDNYGVAMEMAEKCCQANAQFAEISENLRARSNKDAKDPTTKNSLETLLYKPVDRVTRSTLVLHDLLKHTPASHPDHPLLQDALRISQNFLSSINEEITPRRQSMTVKKGEHRQLLKDSFMVELVEGARKLRHVFLFTDLLLCTKLKKQSGGKTQQYDCKWYIPLTDLSFQMVDELEAVPNIPLVPDEELDALKIKISQIKNDIQREKRANKGSKATERLKKKLSEQESLLLLMSPSMAFRVHSRNGKSYTFLISSDYERAEWRENIREQQKKCFRSFSLTSVELQMLTNSCVKLQTVHSIPLTINKEDDESPGLYGFLNVIVHSATGFKQSSNLYCTLEVDSFGYFVNKAKTRVYRDTAEPNWNEEFEIELEGSQTLRILCYEKCYNKTKIPKEDGESTDRLMGKGQVQLDPQALQDRDWQRTVIAMNGIEVKLSVKFNSREFSLKRMPSRKQTGVFGVKIAVVTKRERSKVPYIVRQCVEEIERRGMEEVGIYRVSGVATDIQALKAAFDVNNKDVSVMMSEMDVNAIAGTLKLYFRELPEPLFTDEFYPNFAEGIALSDPVAKESCMLNLLLSLPEANLLTFLFLLDHLKRVAEKEAVNKMSLHNLATVFGPTLLRPSEKESKLPANPSQPITMTDSWSLEVMSQVQVLLYFLQLEAIPAPDSKRQSILFSTEV",
# ABR_HUMAN
"MEPLSHRGLPRLSWIDTLYSNFSYGTDEYDGEGNEEQKGPPEGSETMPYIDESPTMSPQLSARSQGGGDGVSPTPPEGLAPGVEAGKGLEMRKLVLSGFLASEEIYINQLEALLLPMKPLKATATTSQPVLTIQQIETIFYKIQDIYEIHKEFYDNLCPKVQQWDSQVTMGHLFQKLASQLGVYKAFVDNYKVALETAEKCSQSNNQFQKISEELKVKGPKDSKDSHTSVTMEALLYKPIDRVTRSTLVLHDLLKHTPVDHPDYPLLQDALRISQNFLSSINEDIDPRRTAVTTPKGETRQLVKDGFLVEVSESSRKLRHVFLFTDVLLCAKLKKTSAGKHQQYDCKWYIPLADLVFPSPEESEASPQVHPFPDHELEDMKMKISALKSEIQKEKANKGQSRAIERLKKKMFENEFLLLLNSPTIPFRIHNRNGKSYLFLLSSDYERSEWREAIQKLQKKDLQAFVLSSVELQVLTGSCFKLRTVHNIPVTSNKDDDESPGLYGFLHVIVHSAKGFKQSANLYCTLEVDSFGYFVSKAKTRVFRDTAEPKWDEEFEIELEGSQSLRILCYEKCYDKTKVNKDNNEIVDKIMGKGQIQLDPQTVETKNWHTDVIEMNGIKVEFSMKFTSRDMSLKRTPSKKQTGVFGVKISVVTKRERSKVPYIVRQCVEEVEKRGIEEVGIYRISGVATDIQALKAVFDANNKDILLMLSDMDINAIAGTLKLYFRELPEPLLTDRLYPAFMEGIALSDPAAKENCMMHLLRSLPDPNLITFLFLLEHLKRVAEKEPINKMSLHNLATVFGPTLLRPSEVESKAHLTSAADIWSHDVMAQVQVLLYYLQHPPISFAELKRNTLYFSTDV",
# B5MCJ9_HUMAN
"XLLRKDCFQGLIQELGQIAKAHETVADATA",
# Q6IBK5_HUMAN
"MAALGPSSQNVTEYVVRVPKNTTKKYNIMAFNAADKVNFATWNQARLERDLSNKKIYQEEEMPESGAGSEFNRKLREEARRKKYGIVLKEFRPEDQPWLLRVNGKSGRKFKGIKKGGVTENTSYYIFTQCPDGAFEAFPVHNWYNFTPLARHRTLTAEEAEEEWERRNKVLNHFSIMQQRRLKDQDQDEDEEEKEKRGRRKASELRIHDLEDDLEMSSDASDASGEEGGRVPKAKKKAPLAKGGRKKKKKKGSDDEAFEDSDDGDFEGQEVDYMSDGSSSSQEEPESKAKAPQQEEGPKGVDEQSDSSEESEEEKPPEEDKEEEEEKKAPTPQEKKRRKDSSEESDSSEESDIDSEASSALFMAKKKTPPKRERKPSGGSSRGNSRPGTPSAEGGSTSSTLRAAASKLEQGKRVSEMPAAKRLRLDTGPQSLSGKSTPQPPSGKTTPNSGDVQVTEDAVRRYLTRKPMTTKDLLKKFQTKKTGLSSEQTVNVLAQILKRLNPERKMINDKMHFSLKD",
# CERT_HUMAN
"MSDNQSWNSSGSEEDPETESGPPVERCGVLSKWTNYIHGWQDRWVVLKNNALSYYKSEDETEYGCRGSICLSKAVITPHDFDECRFDISVNDSVWYLRAQDPDHRQQWIDAIEQHKTESGYGSESSLRRHGSMVSLVSGASGYSATSTSSFKKGHSLREKLAEMETFRDILCRQVDTLQKYFDACADAVSKDELQRDKVVEDDEDDFPTTRSDGDFLHSTNGNKEKLFPHVTPKGINGIDFKGEAITFKATTAGILATLSHCIELMVKREDSWQKRLDKETEKKRRTEEAYKNAMTELKKKSHFGGPDYEEGPNSLINEEEFFDAVEAALDRQDKIEEQSQSEKVRLHWPTSLPSGDAFSSVGTHRFVQKPYSRSSSMSSIDLVSASDDVHRFSSQVEEMVQNHMTYSLQDVGGDANWQLVVEEGEMKVYRREVEENGIVLDPLKATHAVKGVTGHEVCNYFWNVDVRNDWETTIENFHVVETLADNAIIIYQTHKRVWPASQRDVLYLSVIRKIPALTENDPETWIVCNFSVDHDSAPLNNRCVRAKINVAMICQTLVSPPEGNQEISRDNILCKITYVANVNPGGWAPASVLRAVAKREYPKFLKRFTSYVQEKTAGKPILF",
# BIEA_HUMAN
"MNAEPERKFGVVVVGVGRAGSVRMRDLRNPHPSSAFLNLIGFVSRRELGSIDGVQQISLEDALSSQEVEVAYICSESSSHEDYIRQFLNAGKHVLVEYPMTLSLAAAQELWELAEQKGKVLHEEHVELLMEEFAFLKKEVVGKDLLKGSLLFTAGPLEEERFGFPAFSGISRLTWLVSLFGELSLVSATLEERKEDQYMKMTVCLETEKKSPLSWIEEKGPGLKRNRYLSFHFKSGSLENVPNVGVNKNIFLKDQNIFVQKLLGQFSEKELAAEKKRILHCLGLAEEIQKYCCSRK",
# BAZ1A_HUMAN
"MPLLHRKPFVRQKPPADLRPDEEVFYCKVTNEIFRHYDDFFERTILCNSLVWSCAVTGRPGLTYQEALESEKKARQNLQSFPEPLIIPVLYLTSLTHRSRLHEICDDIFAYVKDRYFVEETVEVIRNNGARLQCRILEVLPPSHQNGFANGHVNSVDGETIIISDSDDSETQSCSFQNGKKKDAIDPLLFKYKVQPTKKELHESAIVKATQISRRKHLFSRDKLKLFLKQHCEPQDGVIKIKASSLSTYKIAEQDFSYFFPDDPPTFIFSPANRRRGRPPKRIHISQEDNVANKQTLASYRSKATKERDKLLKQEEMKSLAFEKAKLKREKADALEAKKKEKEDKEKKREELKKIVEEERLKKKEEKERLKVEREKEREKLREEKRKYVEYLKQWSKPREDMECDDLKELPEPTPVKTRLPPEIFGDALMVLEFLNAFGELFDLQDEFPDGVTLEVLEEALVGNDSEGPLCELLFFFLTAIFQAIAEEEEEVAKEQLTDADTKDLTEALDEDADPTKSALSAVASLAAAWPQLHQGCSLKSLDLDSCTLSEILRLHILASGADVTSANAKYRYQKRGGFDATDDACMELRLSNPSLVKKLSSTSVYDLTPGEKMKILHALCGKLLTLVSTRDFIEDYVDILRQAKQEFRELKAEQHRKEREEAAARIRKRKEEKLKEQEQKMKEKQEKLKEDEQRNSTADISIGEEEREDFDTSIESKDTEQKELDQDMVTEDEDDPGSHKRGRRGKRGQNGFKEFTRQEQINCVTREPLTADEEEALKQEHQRKEKELLEKIQSAIACTNIFPLGRDRMYRRYWIFPSIPGLFIEEDYSGLTEDMLLPRPSSFQNNVQSQDPQVSTKTGEPLMSESTSNIDQGPRDHSVQLPKPVHKPNRWCFYSSCEQLDQLIEALNSRGHRESALKETLLQEKSRICAQLARFSEEKFHFSDKPQPDSKPTYSRGRSSNAYDPSQMCAEKQLELRLRDFLLDIEDRIYQGTLGAIKVTDRHIWRSALESGRYELLSEENKENGIIKTVNEDVEEMEIDEQTKVIVKDRLLGIKTETPSTVSTNASTPQSVSSVVHYLAMALFQIEQGIERRFLKAPLDASDSGRSYKTVLDRWRESLLSSASLSQVFLHLSTLDRSVIWSKSILNARCKICRKKGDAENMVLCDGCDRGHHTYCVRPKLKTVPEGDWFCPECRPKQRSRRLSSRQRPSLESDEDVEDSMGGEDDEVDGDEEEGQSEEEEYEVEQDEDDSQEEEEVSLPKRGRPQVRLPVKTRGKLSSSFSSRGQQQEPGRYPSRSQQSTPKTTVSSKTGRSLRKINSAPPTETKSLRIASRSTRHSHGPLQADVFVELLSPRRKRRGRKSANNTPENSPNFPNFRVIATKSSEQSRSVNIASKLSLQESESKRRCRKRQSPEPSPVTLGRRSSGRQGGVHELSAFEQLVVELVRHDDSWPFLKLVSKIQVPDYYDIIKKPIALNIIREKVNKCEYKLASEFIDDIELMFSNCFEYNPRNTSEAKAGTRLQAFFHIQAQKLGLHVTPSNVDQVSTPPAAKKSRI",
# BAZ1B_HUMAN
"MAPLLGRKPFPLVKPLPGEEPLFTIPHTQEAFRTREEYEARLERYSERIWTCKSTGSSQLTHKEAWEEEQEVAELLKEEFPAWYEKLVLEMVHHNTASLEKLVDTAWLEIMTKYAVGEECDFEVGKEKMLKVKIVKIHPLEKVDEEATEKKSDGACDSPSSDKENSSQIAQDHQKKETVVKEDEGRRESINDRARRSPRKLPTSLKKGERKWAPPKFLPHKYDVKLQNEDKIISNVPADSLIRTERPPNKEIVRYFIRHNALRAGTGENAPWVVEDELVKKYSLPSKFSDFLLDPYKYMTLNPSTKRKNTGSPDRKPSKKSKTDNSSLSSPLNPKLWCHVHLKKSLSGSPLKVKNSKNSKSPEEHLEEMMKMMSPNKLHTNFHIPKKGPPAKKPGKHSDKPLKAKGRSKGILNGQKSTGNSKSPKKGLKTPKTKMKQMTLLDMAKGTQKMTRAPRNSGGTPRTSSKPHKHLPPAALHLIAYYKENKDREDKRSALSCVISKTARLLSSEDRARLPEELRSLVQKRYELLEHKKRWASMSEEQRKEYLKKKREELKKKLKEKAKERREKEMLERLEKQKRYEDQELTGKNLPAFRLVDTPEGLPNTLFGDVAMVVEFLSCYSGLLLPDAQYPITAVSLMEALSADKGGFLYLNRVLVILLQTLLQDEIAEDYGELGMKLSEIPLTLHSVSELVRLCLRRSDVQEESEGSDTDDNKDSAAFEDNEVQDEFLEKLETSEFFELTSEEKLQILTALCHRILMTYSVQDHMETRQQMSAELWKERLAVLKEENDKKRAEKQKRKEMEAKNKENGKVENGLGKTDRKKEIVKFEPQVDTEAEDMISAVKSRRLLAIQAKKEREIQEREMKVKLERQAEEERIRKHKAAAEKAFQEGIAKAKLVMRRTPIGTDRNHNRYWLFSDEVPGLFIEKGWVHDSIDYRFNHHCKDHTVSGDEDYCPRSKKANLGKNASMNTQHGTATEVAVETTTPKQGQNLWFLCDSQKELDELLNCLHPQGIRESQLKERLEKRYQDIIHSIHLARKPNLGLKSCDGNQELLNFLRSDLIEVATRLQKGGLGYVEETSEFEARVISLEKLKDFGECVIALQASVIKKFLQGFMAPKQKRRKLQSEDSAKTEEVDEEKKMVEEAKVASALEKWKTAIREAQTFSRMHVLLGMLDACIKWDMSAENARCKVCRKKGEDDKLILCDECNKAFHLFCLRPALYEVPDGEWQCPACQPATARRNSRGRNYTEESASEDSEDDESDEEEEEEEEEEEEEDYEVAGLRLRPRKTIRGKHSVIPPAARSGRRPGKKPHSTRRSQPKAPPVDDAEVDELVLQTKRSSRRQSLELQKCEEILHKIVKYRFSWPFREPVTRDEAEDYYDVITHPMDFQTVQNKCSCGSYRSVQEFLTDMKQVFTNAEVYNCRGSHVLSCMVKTEQCLVALLHKHLPGHPYVRRKRKKFPDRLAEDEGDSEPEAVGQSRGRRQKK"
]


# Add protein names corresponding to the sequences
protein_names = [
"ABL1_HUMAN",
"ACK1_HUMAN",
"AVR2A_HUMAN",
"AVR2B_HUMAN",
"COQ8B_HUMAN",
"TRIB1_HUMAN",
"BRSK2_HUMAN",
"WNK2_HUMAN",
"AKT1_HUMAN",
"AKT2_HUMAN",
"AKT3_HUMAN",
"KCC1G_HUMAN",
"ALK_HUMAN",
"ACVL1_HUMAN",
"ACVR1_HUMAN",
"BMR1A_HUMAN",
"ACV1B_HUMAN",
"TGFR1_HUMAN",
"BMR1B_HUMAN",
"AAPK1_HUMAN",
"AAPK2_HUMAN",
"ANPRA_HUMAN",
"ANPRB_HUMAN",
"ARAF_HUMAN",
"ABL2_HUMAN",
"ATM_HUMAN",
"ATR_HUMAN",
"AURKC_HUMAN",
"UFO_HUMAN",
"ARBK1_HUMAN",
"BCKD_HUMAN",
"M4K2_HUMAN",
"BLK_HUMAN",
"BRAF_HUMAN",
"PTK6_HUMAN",
"BTK_HUMAN",
"BUB1B_HUMAN",
"CDK7_HUMAN",
"KCC1A_HUMAN",
"KCC2A_HUMAN",
"KCC2B_HUMAN",
"KCC2G_HUMAN",
"KCC4_HUMAN",
"CAMKV_HUMAN",
"DCLK1_HUMAN",
"CSKP_HUMAN",
"CDK1_HUMAN",
"CDC7_HUMAN",
"CDK2_HUMAN",
"CDK3_HUMAN",
"CDK4_HUMAN",
"CDK5_HUMAN",
"CDK6_HUMAN",
"CDK8_HUMAN",
"KGP1_HUMAN",
"KGP2_HUMAN",
"CDK13_HUMAN",
"CHK1_HUMAN",
"CHK2_HUMAN",
"KC1A_HUMAN",
"KC1D_HUMAN",
"KC1E_HUMAN",
"KC1G2_HUMAN",
"KC1G3_HUMAN",
"CSK21_HUMAN",
"CSK22_HUMAN",
"CLK1_HUMAN",
"CLK2_HUMAN",
"CLK3_HUMAN",
"M3K8_HUMAN",
"CSF1R_HUMAN",
"CSK_HUMAN",
"MARK3_HUMAN",
"GUC2D_HUMAN",
"GUC2F_HUMAN",
"DAPK1_HUMAN",
"DAPK2_HUMAN",
"M3K12_HUMAN",
"DMPK_HUMAN",
"MRCKG_HUMAN",
"PRKDC_HUMAN",
"DYR1B_HUMAN",
"DYRK2_HUMAN",
"DYRK4_HUMAN",
"EF2K_HUMAN",
"EGFR_HUMAN",
"E2AK2_HUMAN",
"MARK2_HUMAN",
"EPHA1_HUMAN",
"EPHA2_HUMAN",
"EPHA3_HUMAN",
"EPHA4_HUMAN",
"EPHA5_HUMAN",
"EPHA8_HUMAN",
"EPHB1_HUMAN",
"EPHB2_HUMAN",
"EPHB3_HUMAN",
"EPHB4_HUMAN",
"EPHB6_HUMAN",
"MK03_HUMAN",
"MK01_HUMAN",
"MK06_HUMAN",
"MK04_HUMAN",
"FAK1_HUMAN",
"FER_HUMAN",
"FES_HUMAN",
"FGFR1_HUMAN",
"FGFR2_HUMAN",
"FGFR3_HUMAN",
"FGFR4_HUMAN",
"FGR_HUMAN",
"FLT3_HUMAN",
"VGFR1_HUMAN",
"VGFR3_HUMAN",
"MTOR_HUMAN",
"FYN_HUMAN",
"GAK_HUMAN",
"GRK4_HUMAN",
"GRK5_HUMAN",
"GRK6_HUMAN",
"TRIB2_HUMAN",
"GSK3A_HUMAN",
"GSK3B_HUMAN",
"HCK_HUMAN",
"ERBB2_HUMAN",
"ERBB3_HUMAN",
"ERBB4_HUMAN",
"HIPK1_HUMAN",
"M4K1_HUMAN",
"GUC2C_HUMAN",
"IGF1R_HUMAN",
"IKKA_HUMAN",
"IKKB_HUMAN",
"ILK_HUMAN",
"INSR_HUMAN",
"IRAK1_HUMAN",
"IRAK2_HUMAN",
"IRAK3_HUMAN",
"ERN1_HUMAN",
"INSRR_HUMAN",
"ITK_HUMAN",
"JAK1_HUMAN",
"JAK2_HUMAN",
"JAK3_HUMAN",
"MK08_HUMAN",
"MK09_HUMAN",
"MK10_HUMAN",
"M4K5_HUMAN",
"IKKE_HUMAN",
"NUAK1_HUMAN",
"MAST3_HUMAN",
"STK36_HUMAN",
"PIM3_HUMAN",
"KIT_HUMAN",
"CDKL2_HUMAN",
"CDKL1_HUMAN",
"KSR1_HUMAN",
"LCK_HUMAN",
"LIMK2_HUMAN",
"STK11_HUMAN",
"LTK_HUMAN",
"LYN_HUMAN",
"MAK_HUMAN",
"MAPK2_HUMAN",
"MAPK3_HUMAN",
"MAPK5_HUMAN",
"MARK1_HUMAN",
"MAST2_HUMAN",
"MP2K1_HUMAN",
"MP2K2_HUMAN",
"MP2K5_HUMAN",
"MP2K6_HUMAN",
"M3K1_HUMAN",
"M3K2_HUMAN",
"M3K3_HUMAN",
"M3K4_HUMAN",
"M3K5_HUMAN",
"MERTK_HUMAN",
"MET_HUMAN",
"AMHR2_HUMAN",
"MP2K7_HUMAN",
"MYLK_HUMAN",
"M3K9_HUMAN",
"M3K10_HUMAN",
"DYR1A_HUMAN",
"MKNK1_HUMAN",
"MKNK2_HUMAN",
"MOS_HUMAN",
"MP2K3_HUMAN",
"MP2K4_HUMAN",
"MRCKB_HUMAN",
"KS6A5_HUMAN",
"KS6A4_HUMAN",
"STK4_HUMAN",
"STK3_HUMAN",
"STK24_HUMAN",
"MUSK_HUMAN",
"PMYT1_HUMAN",
"STK38_HUMAN",
"NEK1_HUMAN",
"NEK2_HUMAN",
"NEK3_HUMAN",
"M3K14_HUMAN",
"NLK_HUMAN",
"NEK4_HUMAN",
"IRAK4_HUMAN",
"PI3R4_HUMAN",
"ROCK2_HUMAN",
"MK14_HUMAN",
"KS6B1_HUMAN",
"KS6B2_HUMAN",
"PAK1_HUMAN",
"PAK2_HUMAN",
"PAK3_HUMAN",
"KPCL_HUMAN",
"CDK16_HUMAN",
"CDK17_HUMAN",
"CDK18_HUMAN",
"PGFRA_HUMAN",
"PGFRB_HUMAN",
"PDPK1_HUMAN",
"PDK1_HUMAN",
"PDK2_HUMAN",
"PDK3_HUMAN",
"PDK4_HUMAN",
"E2AK3_HUMAN",
"CDK14_HUMAN",
"PHKG1_HUMAN",
"PHKG2_HUMAN",
"PIM1_HUMAN",
"PIM2_HUMAN",
"CDK10_HUMAN",
"CDK9_HUMAN",
"CD11B_HUMAN",
"MELK_HUMAN",
"MRCKA_HUMAN",
"KAPCA_HUMAN",
"KAPCB_HUMAN",
"KAPCG_HUMAN",
"KPCA_HUMAN",
"KPCB_HUMAN",
"KPCD_HUMAN",
"KPCE_HUMAN",
"KPCG_HUMAN",
"KPCI_HUMAN",
"KPCD1_HUMAN",
"KPCT_HUMAN",
"KPCZ_HUMAN",
"PRKX_HUMAN",
"HIPK3_HUMAN",
"PLK1_HUMAN",
"PLK3_HUMAN",
"PKN1_HUMAN",
"PKN2_HUMAN",
"PRKY_HUMAN",
"PRP4K_HUMAN",
"KPSH1_HUMAN",
"RAF1_HUMAN",
"RET_HUMAN",
"GRK1_HUMAN",
"RIPK1_HUMAN",
"RIPK2_HUMAN",
"RIPK3_HUMAN",
"ROCK1_HUMAN",
"RON_HUMAN",
"ROR1_HUMAN",
"ROR2_HUMAN",
"ROS1_HUMAN",
"KS6A2_HUMAN",
"KS6A3_HUMAN",
"KS6A1_HUMAN",
"RYK_HUMAN",
"PLK4_HUMAN",
"MK11_HUMAN",
"MK12_HUMAN",
"MK13_HUMAN",
"MAST1_HUMAN",
"SGK1_HUMAN",
"SLK_HUMAN",
"PLK2_HUMAN",
"M3K11_HUMAN",
"SRC_HUMAN",
"SRPK1_HUMAN",
"SRPK2_HUMAN",
"CDKL5_HUMAN",
"TAOK2_HUMAN",
"KSYK_HUMAN",
"M3K7_HUMAN",
"BMPR2_HUMAN",
"TEC_HUMAN",
"TIE2_HUMAN",
"TESK1_HUMAN",
"TGFR2_HUMAN",
"TIE1_HUMAN",
"TLK1_HUMAN",
"TLK2_HUMAN",
"TNK1_HUMAN",
"TRIO_HUMAN",
"NTRK1_HUMAN",
"NTRK2_HUMAN",
"NTRK3_HUMAN",
"TRRAP_HUMAN",
"TTK_HUMAN",
"TXK_HUMAN",
"TYK2_HUMAN",
"TYRO3_HUMAN",
"ULK1_HUMAN",
"ULK2_HUMAN",
"VRK1_HUMAN",
"VRK2_HUMAN",
"WEE1_HUMAN",
"YES_HUMAN",
"STK25_HUMAN",
"ZAP70_HUMAN",
"M3K13_HUMAN",
"DDR1_HUMAN",
"ADCK1_HUMAN",
"VGFR2_HUMAN",
"ACV1C_HUMAN",
"AURKB_HUMAN",
"AURKA_HUMAN",
"MK07_HUMAN",
"BUB1_HUMAN",
"DDR2_HUMAN",
"PTK7_HUMAN",
"LIMK1_HUMAN",
"LMTK1_HUMAN",
"LMTK2_HUMAN",
"LMTK3_HUMAN",
"EPHA7_HUMAN",
"BMX_HUMAN",
"MATK_HUMAN",
"FRK_HUMAN",
"NEK6_HUMAN",
"NEK7_HUMAN",
"AAK1_HUMAN",
"TRPM7_HUMAN",
"FAK2_HUMAN",
"SRMS_HUMAN",
"STK10_HUMAN",
"M4K3_HUMAN",
"OXSR1_HUMAN",
"PAK6_HUMAN",
"PAK4_HUMAN",
"STK26_HUMAN",
"STK39_HUMAN",
"STRAA_HUMAN",
"STRAB_HUMAN",
"TAOK3_HUMAN",
"TAOK1_HUMAN",
"M4K4_HUMAN",
"TNIK_HUMAN",
"MINK1_HUMAN",
"NRK_HUMAN",
"LATS1_HUMAN",
"LATS2_HUMAN",
"CDK19_HUMAN",
"NIM1_HUMAN",
"ULK3_HUMAN",
"PDK1L_HUMAN",
"TTBK2_HUMAN",
"SCYL1_HUMAN",
"GWL_HUMAN",
"PINK1_HUMAN",
"ULK4_HUMAN",
"MLKL_HUMAN",
"DCLK3_HUMAN",
"PKDCC_HUMAN",
"CDK15_HUMAN",
"STK33_HUMAN",
"PRPK_HUMAN",
"MK15_HUMAN",
"CDKL4_HUMAN",
"PACE1_HUMAN",
"ST32C_HUMAN",
"NEK9_HUMAN",
"TSSK3_HUMAN",
"NUAK2_HUMAN",
"RPKL1_HUMAN",
"TSSK2_HUMAN",
"SCYL2_HUMAN",
"NEK8_HUMAN",
"ARBK2_HUMAN",
"NRBP_HUMAN",
"KPCD2_HUMAN",
"ST32B_HUMAN",
"KKCC2_HUMAN",
"CDK20_HUMAN",
"CLK4_HUMAN",
"CDK12_HUMAN",
"ST17A_HUMAN",
"ST17B_HUMAN",
"DYRK3_HUMAN",
"KPCD3_HUMAN",
"E2AK4_HUMAN",
"KS6R_HUMAN",
"STK40_HUMAN",
"STK35_HUMAN",
"TNI3K_HUMAN",
"HIPK2_HUMAN",
"E2AK1_HUMAN",
"CILK1_HUMAN",
"ERN2_HUMAN",
"PASK_HUMAN",
"ST38L_HUMAN",
"SIK3_HUMAN",
"HUNK_HUMAN",
"M3K6_HUMAN",
"M3K20_HUMAN",
"MOK_HUMAN",
"STK16_HUMAN",
"SRPK3_HUMAN",
"WNK1_HUMAN",
"CDKL3_HUMAN",
"PAK5_HUMAN",
"PKN3_HUMAN",
"SIK2_HUMAN",
"MARK4_HUMAN",
"DUSTY_HUMAN",
"KS6C1_HUMAN",
"KS6A6_HUMAN",
"NRBP2_HUMAN",
"STKL1_HUMAN",
"SGK2_HUMAN",
"TSSK6_HUMAN",
"SGK3_HUMAN",
"TTBK1_HUMAN",
"DCLK2_HUMAN",
"PXK_HUMAN",
"TOPK_HUMAN",
"STYK1_HUMAN",
"TBK1_HUMAN",
"TESK2_HUMAN",
"KALRN_HUMAN",
"TSSK4_HUMAN",
"VRK3_HUMAN",
"MYLK3_HUMAN",
"SPEG_HUMAN",
"KC1AL_HUMAN",
"RIPK4_HUMAN",
"NEK5_HUMAN",
"KCC1D_HUMAN",
"M3K19_HUMAN",
"NEK11_HUMAN",
"GRK7_HUMAN",
"SBK2_HUMAN",
"HIPK4_HUMAN",
"MYO3B_HUMAN",
"WNK4_HUMAN",
"SBK3_HUMAN",
"BRSK1_HUMAN",
"OBSCN_HUMAN",
"PSKH2_HUMAN",
"SIK1_HUMAN",
"KSR2_HUMAN",
"RIOK3_HUMAN",
"COQ8A_HUMAN",
"RIOK1_HUMAN",
"ST32A_HUMAN",
"SNRK_HUMAN",
"EPHAA_HUMAN",
"SG196_HUMAN",
"MYO3A_HUMAN",
"WNK3_HUMAN",
"PRAG1_HUMAN",
"NEK10_HUMAN",
"EPHA6_HUMAN",
"KC1G1_HUMAN",
"PEAK1_HUMAN",
"SBK1_HUMAN",
"STK31_HUMAN",
"ANKK1_HUMAN",
"UHMK1_HUMAN",
"KCC1B_HUMAN",
"TBCK_HUMAN",
"SMG1_HUMAN",
"MYLK2_HUMAN",
"M3K15_HUMAN",
"LRRK2_HUMAN",
"M3K21_HUMAN",
"HASP_HUMAN",
"TRIB3_HUMAN",
"CTRO_HUMAN",
"KKCC1_HUMAN",
"LRRK1_HUMAN",
"TEX14_HUMAN",
"MAST4_HUMAN",
"KCC2D_HUMAN",
"BMP2K_HUMAN",
"TSSK1_HUMAN",
"MYLK4_HUMAN",
"ADCK2_HUMAN",
"DAPK3_HUMAN",
"WEE2_HUMAN",
"RN5A_HUMAN",
"TRPM6_HUMAN",
"RIOK2_HUMAN",
"ALPK2_HUMAN",
"ALPK1_HUMAN",
"BRD2_HUMAN",
"BRD3_HUMAN",
"BRD4_HUMAN",
"BRDT_HUMAN",
"ALPK3_HUMAN",
"ADCK5_HUMAN",
"TIF1A_HUMAN",
"TIF1B_HUMAN",
"TRI33_HUMAN",
"PAN3_HUMAN",
"FASTK_HUMAN",
"WHR1_HUMAN",
"TAF1_HUMAN",
"TAF1L_HUMAN",
"BCR_HUMAN",
"ABR_HUMAN",
"B5MCJ9_HUMAN",
"Q6IBK5_HUMAN",
"CERT_HUMAN",
"BIEA_HUMAN",
"BAZ1A_HUMAN",
"BAZ1B_HUMAN"
]

In [73]:
#ER
sequences = [
# TM129_HUMAN
"MDSPEVTFTLAYLVFAVCFVFTPNEFHAAGLTVQNLLSGWLGSEDAAFVPFHLRRTAATLLCHSLLPLGYYVGMCLAASEKRLHALSQAPEAWRLFLLLAVTLPSIACILIYYWSRDRWACHPLARTLALYALPQSGWQAVASSVNTEFRRIDKFATGAPGARVIVTDTWVMKVTTYRVHVAQQQDVHLTVTESRQHELSPDSNLPVQLLTIRVASTNPAVQAFDIWLNSTEYGELCEKLRAPIRRAAHVVIHQSLGDLFLETFASLVEVNPAYSVPSSQELEACIGCMQTRASVKLVKTCQEAATGECQQCYCRPMWCLTCMGKWFASRQDPLRPDTWLASRVPCPTCRARFCILDVCTVR",
# ELOV7_HUMAN
"MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVYFVTSLGPKLMENRKPFELKKAMITYNFFIVLFSVYMCYEFVMSGWGIGYSFRCDIVDYSRSPTALRMARTCWLYYFSKFIELLDTIFFVLRKKNSQVTFLHVFHHTIMPWTWWFGVKFAAGGLGTFHALLNTAVHVVMYSYYGLSALGPAYQKYLWWKKYLTSLQLVQFVIVAIHISQFFFMEDCKYQFPVFACIIMSYSFMFLLLFLHFWYRAYTKGQRLPKTVKNGTCKNKDN",
# NBAS_HUMAN
"MAAPESGPALSPGTAEGEEETILYDLLVNTEWPPETEVQPRGNQKHGASFIITKAIRDRLLFLRQYIWYSPAPFLLPDGLVRLVNKQINWHLVLASNGKLLAAVQDQCVEIRSAKDDFTSIIGKCQVPKDPKPQWRRVAWSYDCTLLAYAESTGTVRVFDLMGSELFVISPASSFIGDLSYAIAGLIFLEYKASAQWSAELLVINYRGELRSYLVSVGTNQSYQESHCFSFSSHYPHGINTAIYHPGHRLLLVGGCETAEVGMSKASSCGLSAWRVLSGSPYYKQVTNGGDGVTAVPKTLGLLRMLSVKFYSRQGQEQDGIFKMSLSPDGMLLAAIHFSGKLSIWAIPSLKQQGEWGQNEQPGYDDLNPDWRLSTEKRKKIKDKESFYPLIDVNWWADSAVTLARCSGALTVSSVKTLKNLLGKSCEWFEPSPQVTATHDGGFLSLECEIKLAPKRSRLETRAGEEDEGEEDSDSDYEISAKARYFGYIKQGLYLVTEMERFAPPRKRPRTITKNYRLVSLRSTTPEELYQRKIESEEYEEALSLAHTYGLDTDLVYQRQWRKSAVNVASIQNYLSKIKKRSWVLHECLERVPENVDAAKELLQYGLKGTDLEALLAIGKGADDGRFTLPGEIDIDSISYEELSPPDEEPAKNKKEKELKKRQELLKLVNFSKLTLEQKELCRCRRKLLTYLDRLATYEEILGVPHASEQRYDAEFFKKFRNQNIVLSARTYAQESNVQALEILFTYHGSDLLPHRLAILSNFPETTSPHEYSVLLPEACFNGDSLMIIPWHEHKHRAKDWCEELACRMVVEPNLQDESEFLYAAQPELLRFRMTQLTVEKVMDWYQTRAEEIEHYARQVDCALSLIRLGMERNIPGLLVLCDNLVTLETLVYEARCDVTLTLKELQQMKDIEKLRLLMNSCSEDKYVTSAYQWMVPFLHRCEKQSPGVANELLKEYLVTLAKGDLKFPLKIFQHSKPDLQQKIIPDQDQLMAIALECIYTCERNDQLCLCYDLLECLPERGYGDKTEATTKLHDMVDQLEQILSVSELLEKHGLEKPISFVKNTQSSSEEARKLMVRLTRHTGRKQPPVSESHWRTLLQDMLTMQQNVYTCLDSDACYEIFTESLLCSSRLENIHLAGQMMHCSACSENPPAGIAHKGKPHYRVSYEKSIDLVLAASREYFNSSTNLTDSCMDLARCCLQLITDRPPAIQEELDLIQAVGCLEEFGVKILPLQVRLCPDRISLIKECISQSPTCYKQSTKLLGLAELLRVAGENPEERRGQVLILLVEQALRFHDYKAASMHCQELMATGYPKSWDVCSQLGQSEGYQDLATRQELMAFALTHCPPSSIELLLAASSSLQTEILYQRVNFQIHHEGGENISASPLTSKAVQEDEVGVPGSNSADLLRWTTATTMKVLSNTTTTTKAVLQAVSDGQWWKKSLTYLRPLQGQKCGGAYQIGTTANEDLEKQGCHPFYESVISNPFVAESEGTYDTYQHVPVESFAEVLLRTGKLAEAKNKGEVFPTTEVLLQLASEALPNDMTLALAYLLALPQVLDANRCFEKQSPSALSLQLAAYYYSLQIYARLAPCFRDKCHPLYRADPKELIKMVTRHVTRHEHEAWPEDLISLTKQLHCYNERLLDFTQAQILQGLRKGVDVQRFTADDQYKRETILGLAETLEESVYSIAISLAQRYSVSRWEVFMTHLEFLFTDSGLSTLEIENRAQDLHLFETLKTDPEAFHQHMVKYIYPTIGGFDHERLQYYFTLLENCGCADLGNCAIKPETHIRLLKKFKVVASGLNYKKLTDENMSPLEALEPVLSSQNILSISKLVPKIPEKDGQMLSPSSLYTIWLQKLFWTGDPHLIKQVPGSSPEWLHAYDVCMKYFDRLHPGDLITVVDAVTFSPKAVTKLSVEARKEMTRKAIKTVKHFIEKPRKRNSEDEAQEAKDSKVTYADTLNHLEKSLAHLETLSHSFILSLKNSEQETLQKYSHLYDLSRSEKEKLHDEAVAICLDGQPLAMIQQLLEVAVGPLDISPKDIVQSAIMKIISALSGGSADLGGPRDPLKVLEGVVAAVHASVDKGEELVSPEDLLEWLRPFCADDAWPVRPRIHVLQILGQSFHLTEEDSKLLVFFRTEAILKASWPQRQVDIADIENEENRYCLFMELLESSHHEAEFQHLVLLLQAWPPMKSEYVITNNPWVRLATVMLTRCTMENKEGLGNEVLKMCRSLYNTKQMLPAEGVKELCLLLLNQSLLLPSLKLLLESRDEHLHEMALEQITAVTTVNDSNCDQELLSLLLDAKLLVKCVSTPFYPRIVDHLLASLQQGRWDAEELGRHLREAGHEAEAGSLLLAVRGTHQAFRTFSTALRAAQHWV",
# PEDS1_HUMAN
"MAGAENWPGQQLELDEDEASCCRWGAQHAGARELAALYSPGKRLQEWCSVILCFSLIAHNLVHLLLLARWEDTPLVILGVVAGALIADFLSGLVHWGADTWGSVELPIVGKAFIRPFREHHIDPTAITRHDFIETNGDNCLVTLLPLLNMAYKFRTHSPEALEQLYPWECFVFCLIIFGTFTNQIHKWSHTYFGLPRWVTLLQDWHVILPRKHHRIHHVSPHETYFCITTGWLNYPLEKIGFWRRLEDLIQGLTGEKPRADDMKWAQKIK",
# UBXN8_HUMAN
"MASRGVVGIFFLSAVPLVCLELRRGIPDIGIKDFLLLCGRILLLLALLTLIISVTTSWLNSFKSPQVYLKEEEEKNEKRQKLVRKKQQEAQGEKASRYIENVLKPHQEMKLRKLEERFYQMTGEAWKLSSGHKLGGDEGTSQTSFETSNREAAKSQNLPKPLTEFPSPAEQPTCKEIPDLPEEPSQTAEEVVTVALRCPSGNVLRRRFLKSYSSQVLFDWMTRIGYHISLYSLSTSFPRRPLAVEGGQSLEDIGITVDTVLILEEKEQTN",
# RN103_HUMAN
"MWLKLFFLLLYFLVLFVLARFFEAIVWYETGIFATQLVDPVALSFKKLKTILECRGLGYSGLPEKKDVRELVEKSGDLMEGELYSALKEEEASESVSSTNFSGEMHFYELVEDTKDGIWLVQVIANDRSPLVGKIHWEKMVKKVSRFGIRTGTFNCSSDPRYCRRRGWVRSTLIMSVPQTSTSKGKVMLKEYSGRKIEVEHIFKWITAHAASRIKTIYNAEHLKEEWNKSDQYWLKIYLFANLDQPPAFFSALSIKFTGRVEFIFVNVENWDNKSYMTDIGIYNMPSYILRTPEGIYRYGNHTGEFISLQAMDSFLRSLQPEVNDLFVLSLVLVNLMAWMDLFITQGATIKRFVVLISTLGTYNSLLIISWLPVLGFLQLPYLDSFYEYSLKLLRYSNTTTLASWVRADWMFYSSHPALFLSTYLGHGLLIDYFEKKRRRNNNNDEVNANNLEWLSSLWDWYTSYLFHPIASFQNFPVESDWDEDPDLFLERLAFPDLWLHPLIPTDYIKNLPMWRFKCLGVQSEEEMSEGSQDTENDSESENTDTLSSEKEVFEDKQSVLHNSPGTASHCDAEACSCANKYCQTSPCERKGRSYGSYNTNEDMEPDWLTWPADMLHCTECVVCLENFENGCLLMGLPCGHVFHQNCIVMWLAGGRHCCPVCRWPSYKKKQPYAQHQPLSNDVPS",
# PGRC1_HUMAN
"MAAEDVVATGADPSDLESGGLLHEIFTSPLNLLLLGLCIFLLYKIVRGDQPAASGDSDDDEPPPLPRLKRRDFTPAELRRFDGVQDPRILMAINGKVFDVTKGRKFYGPEGPYGVFAGRDASRGLATFCLDKEALKDEYDDLSDLTAAQQETLSDWESQFTFKYHHVGKLLKEGEEPTVYSDEEEPKDESARKND",
# DNM1L_HUMAN
"MEALIPVINKLQDVFNTVGADIIQLPQIVVVGTQSSGKSSVLESLVGRDLLPRGTGIVTRRPLILQLVHVSQEDKRKTTGEENGVEAEEWGKFLHTKNKLYTDFDEIRQEIENETERISGNNKGVSPEPIHLKIFSPNVVNLTLVDLPGMTKVPVGDQPKDIELQIRELILRFISNPNSIILAVTAANTDMATSEALKISREVDPDGRRTLAVITKLDLMDAGTDAMDVLMGRVIPVKLGIIGVVNRSQLDINNKKSVTDSIRDEYAFLQKKYPSLANRNGTKYLARTLNRLLMHHIRDCLPELKTRINVLAAQYQSLLNSYGEPVDDKSATLLQLITKFATEYCNTIEGTAKYIETSELCGGARICYIFHETFGRTLESVDPLGGLNTIDILTAIRNATGPRPALFVPEVSFELLVKRQIKRLEEPSLRCVELVHEEMQRIIQHCSNYSTQELLRFPKLHDAIVEVVTCLLRKRLPVTNEMVHNLVAIELAYINTKHPDFADACGLMNNNIEEQRRNRLARELPSAVSRDKSSKVPSALAPASQEPSPAASAEADGKLIQDSRRETKNVASGGGGVGDGVQEPTTGNWRGMLKTSKAEELLAEEKSKPIPIMPASPQKGHAVNLLDVPVPVARKLSAREQRDCEVIERLIKSYFLIVRKNIQDSVPKAVMHFLVNHVKDTLQSELVGQLYKSSLLDDLLTESEDMAQRRKEAADMLKALQGASQIIAEIRETHLW",
# PLOD2_HUMAN
"MGGCTVKPQLLLLALVLHPWNPCLGADSEKPSSIPTDKLLVITVATKESDGFHRFMQSAKYFNYTVKVLGQGEEWRGGDGINSIGGGQKVRLMKEVMEHYADQDDLVVMFTECFDVIFAGGPEEVLKKFQKANHKVVFAADGILWPDKRLADKYPVVHIGKRYLNSGGFIGYAPYVNRIVQQWNLQDNDDDQLFYTKVYIDPLKREAINITLDHKCKIFQTLNGAVDEVVLKFENGKARAKNTFYETLPVAINGNGPTKILLNYFGNYVPNSWTQDNGCTLCEFDTVDLSAVDVHPNVSIGVFIEQPTPFLPRFLDILLTLDYPKEALKLFIHNKEVYHEKDIKVFFDKAKHEIKTIKIVGPEENLSQAEARNMGMDFCRQDEKCDYYFSVDADVVLTNPRTLKILIEQNRKIIAPLVTRHGKLWSNFWGALSPDGYYARSEDYVDIVQGNRVGVWNVPYMANVYLIKGKTLRSEMNERNYFVRDKLDPDMALCRNAREMGVFMYISNRHEFGRLLSTANYNTSHYNNDLWQIFENPVDWKEKYINRDYSKIFTENIVEQPCPDVFWFPIFSEKACDELVEEMEHYGKWSGGKHHDSRISGGYENVPTDDIHMKQVDLENVWLHFIREFIAPVTLKVFAGYYTKGFALLNFVVKYSPERQRSLRPHHDASTFTINIALNNVGEDFQGGGCKFLRYNCSIESPRKGWSFMHPGRLTHLHEGLPVKNGTRYIAVSFIDP",
# SCD_HUMAN
"MPAHLLQDDISSSYTTTTTITAPPSRVLQNGGDKLETMPLYLEDDIRPDIKDDIYDPTYKDKEGPSPKVEYVWRNIILMSLLHLGALYGITLIPTCKFYTWLWGVFYYFVSALGITAGAHRLWSHRSYKARLPLRLFLIIANTMAFQNDVYEWARDHRAHHKFSETHADPHNSRRGFFFSHVGWLLVRKHPAVKEKGSTLDLSDLEAEKLVMFQRRYYKPGLLMMCFILPTLVPWYFWGETFQNSVFVATFLRYAVVLNATWLVNSAAHLFGYRPYDKNISPRENILVSLGAVGEGFHNYHHSFPYDYSASEYRWHINFTTFFIDCMAALGLAYDRKKVSKAAILARIKRTGDGNYKSG",
# TOR1A_HUMAN
"MKLGRAVLGLLLLAPSVVQAVEPISLGLALAGVLTGYIYPRLYCLFAECCGQKRSLSREALQKDLDDNLFGQHLAKKIILNAVFGFINNPKPKKPLTLSLHGWTGTGKNFVSKIIAENIYEGGLNSDYVHLFVATLHFPHASNITLYKDQLQLWIRGNVSACARSIFIFDEMDKMHAGLIDAIKPFLDYYDLVDGVSYQKAMFIFLSNAGAERITDVALDFWRSGKQREDIKLKDIEHALSVSVFNNKNSGFWHSSLIDRNLIDYFVPFLPLEYKHLKMCIRVEMQSRGYEIDEDIVSRVAEEMTFFPKEERVFSDKGCKTVFTKLDYYYDD",
# EI24_HUMAN
"MADSVKTFLQDLARGIKDSIWGICTISKLDARIQQKREEQRRRRASSVLAQRRAQSIERKQESEPRIVSRIFQCCAWNGGVFWFSLLLFYRVFIPVLQSVTARIIGDPSLHGDVWSWLEFFLTSIFSALWVLPLFVLSKVVNAIWFQDIADLAFEVSGRKPHPFPSVSKIIADMLFNLLLQALFLIQGMFVSLFPIHLVGQLVSLLHMSLLYSLYCFEYRWFNKGIEMHQRLSNIERNWPYYFGFGLPLAFLTAMQSSYIISGCLFSILFPLFIISANEAKTPGKAYLFQLRLFSLVVFLSNRLFHKTVYLQSALSSSTSAEKFPSPHPSPAKLKATAGH",
# RGRF2_HUMAN
"MQKSVRYNEGHALYLAFLARKEGTKRGFLSKKTAEASRWHEKWFALYQNVLFYFEGEQSCRPAGMYLLEGCSCERTPAPPRAGAGQGGVRDALDKQYYFTVLFGHEGQKPLELRCEEEQDGKEWMEAIHQASYADILIEREVLMQKYIHLVQIVETEKIAANQLRHQLEDQDTEIERLKSEIIALNKTKERMRPYQSNQEDEDPDIKKIKKVQSFMRGWLCRRKWKTIVQDYICSPHAESMRKRNQIVFTMVEAESEYVHQLYILVNGFLRPLRMAASSKKPPISHDDVSSIFLNSETIMFLHEIFHQGLKARIANWPTLILADLFDILLPMLNIYQEFVRNHQYSLQVLANCKQNRDFDKLLKQYEANPACEGRMLETFLTYPMFQIPRYIITLHELLAHTPHEHVERKSLEFAKSKLEELSRVMHDEVSDTENIRKNLAIERMIVEGCDILLDTSQTFIRQGSLIQVPSVERGKLSKVRLGSLSLKKEGERQCFLFTKHFLICTRSSGGKLHLLKTGGVLSLIDCTLIEEPDASDDDSKGSGQVFGHLDFKIVVEPPDAAAFTVVLLAPSRQEKAAWMSDISQCVDNIRCNGLMTIVFEENSKVTVPHMIKSDARLHKDDTDICFSKTLNSCKVPQIRYASVERLLERLTDLRFLSIDFLNTFLHTYRIFTTAAVVLGKLSDIYKRPFTSIPVRSLELFFATSQNNRGEHLVDGKSPRLCRKFSSPPPLAVSRTSSPVRARKLSLTSPLNSKIGALDLTTSSSPTTTTQSPAASPPPHTGQIPLDLSRGLSSPEQSPGTVEENVDNPRVDLCNKLKRSIQKAVLESAPADRAGVESSPAADTTELSPCRSPSTPRHLRYRQPGGQTADNAHCSVSPASAFAIATAAAGHGSPPGFNNTERTCDKEFIIRRTATNRVLNVLRHWVSKHAQDFELNNELKMNVLNLLEEVLRDPDLLPQERKAAANILRALSQDDQDDIHLKLEDIIQMTDCMKAECFESLSAMELAEQITLLDHVIFRSIPYEEFLGQGWMKLDKNERTPYIMKTSQHFNDMSNLVASQIMNYADVSSRANAIEKWVAVADICRCLHNYNGVLEITSALNRSAIYRLKKTWAKVSKQTKALMDKLQKTVSSEGRFKNLRETLKNCNPPAVPYLGMYLTDLAFIEEGTPNFTEEGLVNFSKMRMISHIIREIRQFQQTSYRIDHQPKVAQYLLDKDLIIDEDTLYELSLKIEPRLPA",
# MGST3_HUMAN
"MAVLSKEYGFVLLTGAASFIMVAHLAINVSKARKKYKVEYPIMYSTDPENGHIFNCIQRAHQNTLEVYPPFLFFLAVGGVYHPRIASGLGLAWIVGRVLYAYGYYTGEPSKRSRGALGSIALLGLVGTTVCSAFQHLGWVKSGLGSGPKCCH",
# CLGN_HUMAN
"MHFQAFWLCLGLLFISINAEFMDDDVETEDFEENSEEIDVNESELSSEIKYKTPQPIGEVYFAETFDSGRLAGWVLSKAKKDDMDEEISIYDGRWEIEELKENQVPGDRGLVLKSRAKHHAISAVLAKPFIFADKPLIVQYEVNFQDGIDCGGAYIKLLADTDDLILENFYDKTSYIIMFGPDKCGEDYKLHFIFRHKHPKTGVFEEKHAKPPDVDLKKFFTDRKTHLYTLVMNPDDTFEVLVDQTVVNKGSLLEDVVPPIKPPKEIEDPNDKKPEEWDERAKIPDPSAVKPEDWDESEPAQIEDSSVVKPAGWLDDEPKFIPDPNAEKPDDWNEDTDGEWEAPQILNPACRIGCGEWKPPMIDNPKYKGVWRPPLVDNPNYQGIWSPRKIPNPDYFEDDHPFLLTSFSALGLELWSMTSDIYFDNFIICSEKEVADHWAADGWRWKIMIANANKPGVLKQLMAAAEGHPWLWLIYLVTAGVPIALITSFCWPRKVKKKHKDTEYKKTDICIPQTKGVLEQEEKEEKAALEKPMDLEEEKKQNDGEMLEKEEESEPEEKSEEEIEIIEGQEESNQSNKSGSEDEMKEADESTGSGDGPIKSVRKRRVRKD",
# SC16A_HUMAN
"MQPPPQTVPSGMAGPPPAGNPRSVFWASSPYRRRANNNAAVAPTTCPLQPVTDPFAFSRQALQSTPLGSSSKSSPPVLQGPAPAGFSQHPGLLVPHTHARDSSQGPCEPLPGPLTQPRAHASPFSGALTPSAPPGPEMNRSAEVGPSSEPEVQTLPYLPHYIPGVDPETSHGGHPHGNMPGLDRPLSRQNPHDGVVTPAASPSLPQPGLQMPGQWGPVQGGPQPSGQHRSPCPEGPVPSGVPCATSVPHFPTPSILHQGPGHEQHSPLVAPPAALPSDGRDEVSHLQSGSHLANNSDPESTFRQNPRIVNHWASPELRQNPGVKNEHRPASALVNPLARGDSPENRTHHPLGAGAGSGCAPLEADSGASGALAMFFQGGETENEENLSSEKAGLSGQADFDDFCSSPGLGRPPAPTHVGAGSLCQALLPGPSNEAAGDVWGDTASTGVPDASGSQYENVENLEFVQNQEVLPSEPLNLDPSSPSDQFRYGPLPGPAVPRHGAVCHTGAPDATLHTVHPDSVSSSYSSRSHGRLSGSARPQELVGTFIQQEVGKPEDEASGSFFKQIDSSPVGGETDETTVSQNYRGSVSQPSTPSPPKPTGIFQTSANSSFEPVKSHLVGVKPFEADRANVVGEVRETCVRQKQCRPAAALPDASPGNLEQPPDNMETLCAPQVCPLPLNSTTEAVHMLPHAGAPPLDTVYPAPEKRPSARTQGPVKCESPATTLWAQSELPDFGGNVLLAPAAPALYVCAKPQPPVVQPPEEAMSGQQSRNPSSAAPVQSRGGIGASENLENPPKMGEEEALQSQASSGYASLLSSPPTESLQNPPVLIAQPDHSYNLAQPINFSVSLSNSHEKNQSWREALVGDRPAVSSWALGGDSGENTSLSGIPTSSVLSLSLPSSVAQSNFPQGSGASEMVSNQPANLLVQPPSQPVPENLVPESQKDRKAGSALPGFANSPAGSTSVVLVPPAHGTLVPDGNKANHSSHQEDTYGALDFTLSRTLENPVNVYNPSHSDSLASQQSVASHPRQSGPGAPNLDRFYQQVTKDAQGQPGLERAQQELVPPQQQASPPQLPKAMFSELSNPESLPAQGQAQNSAQSPASLVLVDAGQQLPPRPPQSSSVSLVSSGSGQAAVPSEQPWPQPVPALAPGPPPQDLAAYYYYRPLYDAYQPQYSLPYPPEPGAASLYYQDVYSLYEPRYRPYDGAASAYAQNYRYPEPERPSSRASHSSERPPPRQGYPEGYYSSKSGWSSQSDYYASYYSSQYDYGDPGHWDRYHYSARVRDPRTYDRRYWCDAEYDAYRREHSAFGDRPEKRDNNWRYDPRFTGSFDDDPDPHRDPYGEEVDRRSVHSEHSARSLHSAHSLASRRSSLSSHSHQSQIYRSHNVAAGSYEAPLPPGSFHGDFAYGTYRSNFSSGPGFPEYGYPADTVWPAMEQVSSRPTSPEKFSVPHVCARFGPGGQLIKVIPNLPSEGQPALVEVHSMEALLQHTSEQEEMRAFPGPLAKDDTHKVDVINFAQNKAMKCLQNENLIDKESASLLWNFIVLLCRQNGTVVGTDIAELLLRDHRTVWLPGKSPNEANLIDFTNEAVEQVEEEESGEAQLSFLTGGPAAAASSLERETERFRELLLYGRKKDALESAMKNGLWGHALLLASKMDSRTHARVMTRFANSLPINDPLQTVYQLMSGRMPAASTCCGDEKWGDWRPHLAMVLSNLNNNMDVESRTMATMGDTLASRGLLDAAHFCYLMAQAGFGVYTKKTTKLVLIGSNHSLPFLKFATNEAIQRTEAYEYAQSLGAETCPLPSFQVFKFIYSCRLAEMGLATQAFHYCEAIAKSILTQPHLYSPVLISQLVQMASQLRLFDPQLKEKPEEESLAAPTWLVHLQQVERQIKEGAGVWHQDGALPQQCPGTPSSEMEQLDRPGLSQPGALGIANPLLAVPAPSPEHSSPSVRLLPSAPQTLPDGPLASPARVPMFPVPLPPGPLEPGPGCVTPGPALGFLEPSGPGLPPGVPPLQERRHLLQEARSPDPGIVPQEAPVGNSLSELSEENFDGKFANLTPSRTVPDSEAPPGWDRADSGPTQPPLSLSPAPETKRPGQAAKKETKEPKKGESWFFRWLPGKKKTEAYLPDDKNKSIVWDEKKNQWVNLNEPEEEKKAPPPPPTSMPKTVQAAPPALPGPPGAPVNMYSRRAAGTRARYVDVLNPSGTQRSEPALAPADFVAPLAPLPIPSNLFVPTPDAEEPQLPDGTGREGPAAARGLANPEPAPEPKVLSSAASLPGSELPSSRPEGSQGGELSRCSSMSSLSREVSQHFNQAPGDLPAAGGPPSGAMPFYNPAQLAQACATSGSSRLGRIGQRKHLVLN",
# PLCB_HUMAN
"MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASLVCLLRHGGRTVENMSIIGWFVRSFKYFYGLRFEVRDPRRLQEARPCVIVSNHQSILDMMGLMEVLPERCVQIAKRELLFLGPVGLIMYLGGVFFINRQRSSTAMTVMADLGERMVRENLKVWIYPEGTRNDNGDLLPFKKGAFYLAVQAQVPIVPVVYSSFSSFYNTKKKFFTSGTVTVQVLEAIPTSGLTAADVPALVDTCHRAMRTTFLHISKTPQENGATAGSGVQPAQ",
# DEGS1_HUMAN
"MGSRVSREDFEWVYTDQPHADRRREILAKYPEIKSLMKPDPNLIWIIIMMVLTQLGAFYIVKDLDWKWVIFGAYAFGSCINHSMTLAIHEIAHNAAFGNCKAMWNRWFGMFANLPIGIPYSISFKRYHMDHHRYLGADGVDVDIPTDFEGWFFCTAFRKFIWVILQPLFYAFRPLFINPKPITYLEVINTVAQVTFDILIYYFLGIKSLVYMLAASLLGLGLHPISGHFIAEHYMFLKGHETYSYYGPLNLLTFNVGYHNEHHDFPNIPGKSLPLVRKIAAEYYDNLPHYNSWIKVLYDFVMDDTISPYSRMKRHQKGEMVLE",
# SPTC1_HUMAN
"MATATEQWVLVEMVQALYEAPAYHLILEGILILWIIRLLFSKTYKLQERSDLTVKEKEELIEEWQPEPLVPPVPKDHPALNYNIVSGPPSHKTVVNGKECINFASFNFLGLLDNPRVKAAALASLKKYGVGTCGPRGFYGTFDVHLDLEDRLAKFMKTEEAIIYSYGFATIASAIPAYSKRGDIVFVDRAACFAIQKGLQASRSDIKLFKHNDMADLERLLKEQEIEDQKNPRKARVTRRFIVVEGLYMNTGTICPLPELVKLKYKYKARIFLEESLSFGVLGEHGRGVTEHYGINIDDIDLISANMENALASIGGFCCGRSFVIDHQRLSGQGYCFSASLPPLLAAAAIEALNIMEENPGIFAVLKEKCGQIHKALQGISGLKVVGESLSPAFHLQLEESTGSREQDVRLLQEIVDQCMNRSIALTQARYLEKEEKCLPPPSIRVVVTVEQTEEELERAASTIKEVAQAVLL",
# GPR37_HUMAN
"MRAPGALLARMSRLLLLLLLKVSASSALGVAPASRNETCLGESCAPTVIQRRGRDAWGPGNSARDVLRARAPREEQGAAFLAGPSWDLPAAPGRDPAAGRGAEASAAGPPGPPTRPPGPWRWKGARGQEPSETLGRGNPTALQLFLQISEEEEKGPRGAGISGRSQEQSVKTVPGASDLFYWPRRAGKLQGSHHKPLSKTANGLAGHEGWTIALPGRALAQNGSLGEGIHEPGGPRRGNSTNRRVRLKNPFYPLTQESYGAYAVMCLSVVIFGTGIIGNLAVMCIVCHNYYMRSISNSLLANLAFWDFLIIFFCLPLVIFHELTKKWLLEDFSCKIVPYIEVASLGVTTFTLCALCIDRFRAATNVQMYYEMIENCSSTTAKLAVIWVGALLLALPEVVLRQLSKEDLGFSGRAPAERCIIKISPDLPDTIYVLALTYDSARLWWYFGCYFCLPTLFTITCSLVTARKIRKAEKACTRGNKRQIQLESQMNCTVVALTILYGFCIIPENICNIVTAYMATGVSQQTMDLLNIISQFLLFFKSCVTPVLLFCLCKPFSRAFMECCCCCCEECIQKSSTVTSDDNDNEYTTELELSPFSTIRREMSTFASVGTHC",
# INSI1_HUMAN
"MPRLHDHFWSCSCAHSARRRGPPRASAAGLAAKVGEMINVSVSGPSLLAAHGAPDADPAPRGRSAAMSGPEPGSPYPNTWHHRLLQRSLVLFSVGVVLALVLNLLQIQRNVTLFPEEVIATIFSSAWWVPPCCGTAAAVVGLLYPCIDSHLGEPHKFKREWASVMRCIAVFVGINHASAKLDFANNVQLSLTLAALSLGLWWTFDRSRSGLGLGITIAFLATLITQFLVYNGVYQYTSPDFLYIRSWLPCIFFSGGVTVGNIGRQLAMGVPEKPHSD",
# TPSN_HUMAN
"MKSLSLLLAVALGLATAVSAGPAVIECWFVEDASGKGLAKRPGALLLRQGPGEPPPRPDLDPELYLSVHDPAGALQAAFRRYPRGAPAPHCEMSRFVPLPASAKWASGLTPAQNCPRALDGAWLMVSISSPVLSLSSLLRPQPEPQQEPVLITMATVVLTVLTHTPAPRVRLGQDALLDLSFAYMPPTSEAASSLAPGPPPFGLEWRRQHLGKGHLLLAATPGLNGQMPAAQEGAVAFAAWDDDEPWGPWTGNGTFWLPTVQPFQEGTYLATIHLPYLQGQVTLELAVYKPPKVSLMPATLARAAPGEAPPELLCLVSHFYPSGGLEVEWELRGGPGGRSQKAEGQRWLSALRHHSDGSVSLSGHLQPPPVTTEQHGARYACRIHHPSLPASGRSAEVTLEVAGLSGPSLEDSVGLFLSAFLLLGLFKALGWAAVYLSTCKDSKKKAE",
# ZW10_HUMAN
"MASFVTEVLAHSGRLEKEDLGTRISRLTRRVEEIKGEVCNMISKKYSEFLPSMQSAQGLITQVDKLSEDIDLLKSRIESEVRRDLHVSTGEFTDLKQQLERDSVVLSLLKQLQEFSTAIEEYNCALTEKKYVTGAQRLEEAQKCLKLLKSRKCFDLKILKSLSMELTIQKQNILYHLGEEWQKLIVWKFPPSKDTSSLESYLQTELHLYTEQSHKEEKTPMPPISSVLLAFSVLGELHSKLKSFGQMLLKYILRPLASCPSLHAVIESQPNIVIIRFESIMTNLEYPSPSEVFTKIRLVLEVLQKQLLDLPLDTDLENEKTSTVPLAEMLGDMIWEDLSECLIKNCLVYSIPTNSSKLQQYEEIIQSTEEFENALKEMRFLKGDTTDLLKYARNINSHFANKKCQDVIVAARNLMTSEIHNTVKIIPDSKINVPELPTPDEDNKLEVQKVSNTQYHEVMNLEPENTLDQHSFSLPTCRISESVKKLMELAYQTLLEATTSSDQCAVQLFYSVRNIFHLFHDVVPTYHKENLQKLPQLAAIHHNNCMYIAHHLLTLGHQFRLRLAPILCDGTATFVDLVPGFRRLGTECFLAQMRAQKGELLERLSSARNFSNMDDEENYSAASKAVRQVLHQLKRLGIVWQDVLPVNIYCKAMGTLLNTAISEVIGKITALEDISTEDGDRLYSLCKTVMDEGPQVFAPLSEESKNKKYQEEVPVYVPKWMPFKELMMMLQASLQEIGDRWADGKGPLAAAFSSSEVKALIRALFQNTERRAAALAKIK",
# HTRA2_HUMAN
"MAAPRAGRGAGWSLRAWRALGGIRWGRRPRLTPDLRALLTSGTSDPRARVTYGTPSLWARLSVGVTEPRACLTSGTPGPRAQLTAVTPDTRTREASENSGTRSRAWLAVALGAGGAVLLLLWGGGRGPPAVLAAVPSPPPASPRSQYNFIADVVEKTAPAVVYIEILDRHPFLGREVPISNGSGFVVAADGLIVTNAHVVADRRRVRVRLLSGDTYEAVVTAVDPVADIATLRIQTKEPLPTLPLGRSADVRQGEFVVAMGSPFALQNTITSGIVSSAQRPARDLGLPQTNVEYIQTDAAIDFGNSGGPLVNLDGEVIGVNTMKVTAGISFAIPSDRLREFLHRGEKKNSSSGISGSQRRYIGVMMLTLSPSILAELQLREPSFPDVQHGVLIHKVILGSPAHRAGLRPGDVILAIGEQMVQNAEDVYEAVRTQSQLAVQIRRGRETLTLYVTPEVTE",
# ERD23_HUMAN
"MNVFRILGDLSHLLAMILLLGKIWRSKCCKGISGKSQILFALVFTTRYLDLFTNFISIYNTVMKVVFLLCAYVTVYMIYGKFRKTFDSENDTFRLEFLLVPVIGLSFLENYSFTLLEILWTFSIYLESVAILPQLFMISKTGEAETITTHYLFFLGLYRALYLANWIRRYQTENFYDQIAVVSGVVQTIFYCDFFYLYVTKVLKGKKLSLPMPI",
# G6PT1_HUMAN
"MAAQGYGYYRTVIFSAMFGGYSLYYFNRKTFSFVMPSLVEEIPLDKDDLGFITSSQSAAYAISKFVSGVLSDQMSARWLFSSGLLLVGLVNIFFAWSSTVPVFAALWFLNGLAQGLGWPPCGKVLRKWFEPSQFGTWWAILSTSMNLAGGLGPILATILAQSYSWRSTLALSGALCVVVSFLCLLLIHNEPADVGLRNLDPMPSEGKKGSLKEESTLQELLLSPYLWVLSTGYLVVFGVKTCCTDWGQFFLIQEKGQSALVGSSYMSALEVGGLVGSIAAGYLSDRAMAKAGLSNYGNPRHGLLLFMMAGMTVSMYLFRVTVTSDSPKLWILVLGAVFGFSSYGPIALFGVIANESAPPNLCGTSHAIVGLMANVGGFLAGLPFSTIAKHYSWSTAFWVAEVICAASTAAFFLLRNIRTKMGRVSKKAE",
# CALU_HUMAN
"MDLRQFLMCLSLCTAFALSKPTEKKDRVHHEPQLSDKVHNDAQSFDYDHDAFLGAEEAKTFDQLTPEESKERLGKIVSKIDGDKDGFVTVDELKDWIKFAQKRWIYEDVERQWKGHDLNEDGLVSWEEYKNATYGYVLDDPDPDDGFNYKQMMVRDERRFKMADKDGDLIATKEEFTAFLHPEEYDYMKDIVVQETMEDIDKNADGFIDLEEYIGDMYSHDGNTDEPEWVKTEREQFVEFRDKNRDGKMDKEETKDWILPSDYDHAEAEARHLVYESDQNKDGKLTKEEIVDKYDLFVGSQATDFGEALVRHDEF",
# CREB3_HUMAN
"MELELDAGDQDLLAFLLEESGDLGTAPDEAVRAPLDWALPLSEVPSDWEVDDLLCSLLSPPASLNILSSSNPCLVHHDHTYSLPRETVSMDLESESCRKEGTQMTPQHMEELAEQEIARLVLTDEEKSLLEKEGLILPETLPLTKTEEQILKRVRRKIRNKRSAQESRRKKKVYVGGLESRVLKYTAQNMELQNKVQLLEEQNLSLLDQLRKLQAMVIEISNKTSSSSTCILVLLVSFCLLLVPAMYSSDTRGSLPAEHGVLSRQLRALPSEDPYQLELPALQSEVPKDSTHQWLDGSDCVLQAPGNTSCLLHYMPQAPSAEPPLEWPFPDLFSEPLCRGPILPLQANLTRKGGWLPTGSPSVILQDRYSG",
# EXTL3_HUMAN
"MTGYTMLRNGGAGNGGQTCMLRWSNRIRLTWLSFTLFVILVFFPLIAHYYLTTLDEADEAGKRIFGPRVGNELCEVKHVLDLCRIRESVSEELLQLEAKRQELNSEIAKLNLKIEACKKSIENAKQDLLQLKNVISQTEHSYKELMAQNQPKLSLPIRLLPEKDDAGLPPPKATRGCRLHNCFDYSRCPLTSGFPVYVYDSDQFVFGSYLDPLVKQAFQATARANVYVTENADIACLYVILVGEMQEPVVLRPAELEKQLYSLPHWRTDGHNHVIINLSRKSDTQNLLYNVSTGRAMVAQSTFYTVQYRPGFDLVVSPLVHAMSEPNFMEIPPQVPVKRKYLFTFQGEKIESLRSSLQEARSFEEEMEGDPPADYDDRIIATLKAVQDSKLDQVLVEFTCKNQPKPSLPTEWALCGEREDRLELLKLSTFALIITPGDPRLVISSGCATRLFEALEVGAVPVVLGEQVQLPYQDMLQWNEAALVVPKPRVTEVHFLLRSLSDSDLLAMRRQGRFLWETYFSTADSIFNTVLAMIRTRIQIPAAPIREEAAAEIPHRSGKAAGTDPNMADNGDLDLGPVETEPPYASPRYLRNFTLTVTDFYRSWNCAPGPFHLFPHTPFDPVLPSEAKFLGSGTGFRPIGGGAGGSGKEFQAALGGNVPREQFTVVMLTYEREEVLMNSLERLNGLPYLNKVVVVWNSPKLPSEDLLWPDIGVPIMVVRTEKNSLNNRFLPWNEIETEAILSIDDDAHLRHDEIMFGFRVWREARDRIVGFPGRYHAWDIPHQSWLYNSNYSCELSMVLTGAAFFHKYYAYLYSYVMPQAIRDMVDEYINCEDIAMNFLVSHITRKPPIKVTSRWTFRCPGCPQALSHDDSHFHERHKCINFFVKVYGYMPLLYTQFRVDSVLFKTRLPHDKTKCFKFI",
# AT10A_HUMAN
"MEREPAGTEEPGPPGRRRRREGRTRTVRSNLLPPPGAEDPAAGAAKGERRRRRGCAQHLADNRLKTTKYTLLSFLPKNLFEQFHRPANVYFVFIALLNFVPAVNAFQPGLALAPVLFILAITAFRDLWEDYSRHRSDHKINHLGCLVFSREEKKYVNRFWKEIHVGDFVRLRCNEIFPADILLLSSSDPDGLCHIETANLDGETNLKRRQVVRGFSELVSEFNPLTFTSVIECEKPNNDLSRFRGCIIHDNGKKAGLYKENLLLRGCTLRNTDAVVGIVIYAGHETKALLNNSGPRYKRSKLERQMNCDVLWCVLLLVCMSLFSAVGHGLWIWRYQEKKSLFYVPKSDGSSLSPVTAAVYSFLTMIIVLQVLIPISLYVSIEIVKACQVYFINQDMQLYDEETDSQLQCRALNITEDLGQIQYIFSDKTGTLTENKMVFRRCTVSGVEYSHDANAQRLARYQEADSEEEEVVPRGGSVSQRGSIGSHQSVRVVHRTQSTKSHRRTGSRAEAKRASMLSKHTAFSSPMEKDITPDPKLLEKVSECDKSLAVARHQEHLLAHLSPELSDVFDFFIALTICNTVVVTSPDQPRTKVRVRFELKSPVKTIEDFLRRFTPSCLTSGCSSIGSLAANKSSHKLGSSFPSTPSSDGMLLRLEERLGQPTSAIASNGYSSQADNWASELAQEQESERELRYEAESPDEAALVYAARAYNCVLVERLHDQVSVELPHLGRLTFELLHTLGFDSVRKRMSVVIRHPLTDEINVYTKGADSVVMDLLQPCSSVDARGRHQKKIRSKTQNYLNVYAAEGLRTLCIAKRVLSKEEYACWLQSHLEAESSLENSEELLFQSAIRLETNLHLLGATGIEDRLQDGVPETISKLRQAGLQIWVLTGDKQETAVNIAYACKLLDHDEEVITLNATSQEACAALLDQCLCYVQSRGLQRAPEKTKGKVSMRFSSLCPPSTSTASGRRPSLVIDGRSLAYALEKNLEDKFLFLAKQCRSVLCCRSTPLQKSMVVKLVRSKLKAMTLAIGDGANDVSMIQVADVGVGISGQEGMQAVMASDFAVPKFRYLERLLILHGHWCYSRLANMVLYFFYKNTMFVGLLFWFQFFCGFSASTMIDQWYLIFFNLLFSSLPPLVTGVLDRDVPANVLLTNPQLYKSGQNMEEYRPRTFWFNMADAAFQSLVCFSIPYLAYYDSNVDLFTWGTPIVTIALLTFLLHLGIETKTWTWLNWITCGFSVLLFFTVALIYNASCATCYPPSNPYWTMQALLGDPVFYLTCLMTPVAALLPRLFFRSLQGRVFPTQLQLARQLTRKSPRRCSAPKETFAQGRLPKDSGTEHSSGRTVKTSVPLSQPSWHTQQPVCSLEASGEPSTVDMSMPVREHTLLEGLSAPAPMSSAPGEAVLRSPGGCPEESKVRAASTGRVTPLSSLFSLPTFSLLNWISSWSLVSRLGSVLQFSRTEQLADGQAGRGLPVQPHSGRSGLQGPDHRLLIGASSRRSQ",
# MARH6_HUMAN
"MDTAEEDICRVCRSEGTPEKPLYHPCVCTGSIKFIHQECLVQWLKHSRKEYCELCKHRFAFTPIYSPDMPSRLPIQDIFAGLVTSIGTAIRYWFHYTLVAFAWLGVVPLTACRIYKCLFTGSVSSLLTLPLDMLSTENLLADCLQGCFVVTCTLCAFISLVWLREQIVHGGAPIWLEHAAPPFNAAGHHQNEAPAGGNGAENVAADQPANPPAENAVVGENPDAQDDQAEEEEEDNEEEDDAGVEDAADANNGAQDDMNWNALEWDRAAEELTWERMLGLDGSLVFLEHVFWVVSLNTLFILVFAFCPYHIGHFSLVGLGFEEHVQASHFEGLITTIVGYILLAITLIICHGLATLVKFHRSRRLLGVCYIVVKVSLLVVVEIGVFPLICGWWLDICSLEMFDATLKDRELSFQSAPGTTMFLHWLVGMVYVFYFASFILLLREVLRPGVLWFLRNLNDPDFNPVQEMIHLPIYRHLRRFILSVIVFGSIVLLMLWLPIRIIKSVLPNFLPYNVMLYSDAPVSELSLELLLLQVVLPALLEQGHTRQWLKGLVRAWTVTAGYLLDLHSYLLGDQEENENSANQQVNNNQHARNNNAIPVVGEGLHAAHQAILQQGGPVGFQPYRRPLNFPLRIFLLIVFMCITLLIASLICLTLPVFAGRWLMSFWTGTAKIHELYTAACGLYVCWLTIRAVTVMVAWMPQGRRVIFQKVKEWSLMIMKTLIVAVLLAGVVPLLLGLLFELVIVAPLRVPLDQTPLFYPWQDWALGVLHAKIIAAITLMGPQWWLKTVIEQVYANGIRNIDLHYIVRKLAAPVISVLLLSLCVPYVIASGVVPLLGVTAEMQNLVHRRIYPFLLMVVVLMAILSFQVRQFKRLYEHIKNDKYLVGQRLVNYERKSGKQGSSPPPPQSSQE",
# PLOD3_HUMAN
"MTSSGPGPRFLLLLPLLLPPAASASDRPRGRDPVNPEKLLVITVATAETEGYLRFLRSAEFFNYTVRTLGLGEEWRGGDVARTVGGGQKVRWLKKEMEKYADREDMIIMFVDSYDVILAGSPTELLKKFVQSGSRLLFSAESFCWPEWGLAEQYPEVGTGKRFLNSGGFIGFATTIHQIVRQWKYKDDDDDQLFYTRLYLDPGLREKLSLNLDHKSRIFQNLNGALDEVVLKFDRNRVRIRNVAYDTLPIVVHGNGPTKLQLNYLGNYVPNGWTPEGGCGFCNQDRRTLPGGQPPPRVFLAVFVEQPTPFLPRFLQRLLLLDYPPDRVTLFLHNNEVFHEPHIADSWPQLQDHFSAVKLVGPEEALSPGEARDMAMDLCRQDPECEFYFSLDADAVLTNLQTLRILIEENRKVIAPMLSRHGKLWSNFWGALSPDEYYARSEDYVELVQRKRVGVWNVPYISQAYVIRGDTLRMELPQRDVFSGSDTDPDMAFCKSFRDKGIFLHLSNQHEFGRLLATSRYDTEHLHPDLWQIFDNPVDWKEQYIHENYSRALEGEGIVEQPCPDVYWFPLLSEQMCDELVAEMEHYGQWSGGRHEDSRLAGGYENVPTVDIHMKQVGYEDQWLQLLRTYVGPMTESLFPGYHTKARAVMNFVVRYRPDEQPSLRPHHDSSTFTLNVALNHKGLDYEGGGCRFLRYDCVISSPRKGWALLHPGRLTHYHEGLPTTWGTRYIMVSFVDP",
# UD19_HUMAN
"MACTGWTSPLPLCVCLLLTCGFAEAGKLLVVPMDGSHWFTMRSVVEKLILRGHEVVVVMPEVSWQLGRSLNCTVKTYSTSYTLEDLDREFKAFAHAQWKAQVRSIYSLLMGSYNDIFDLFFSNCRSLFKDKKLVEYLKESSFDAVFLDPFDNCGLIVAKYFSLPSVVFARGILCHYLEEGAQCPAPLSYVPRILLGFSDAMTFKERVRNHIMHLEEHLLCHRFFKNALEIASEILQTPVTEYDLYSHTSIWLLRTDFVLDYPKPVMPNMIFIGGINCHQGKPLPMEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# ICMT_HUMAN
"MAGCAARAPPGSEARLSLATFLLGASVLALPLLTRAGLQGRTGLALYVAGLNALLLLLYRPPRYQIAIRACFLGFVFGCGTLLSFSQSSWSHFGWYMCSLSLFHYSEYLVTAVNNPKSLSLDSFLLNHSLEYTVAALSSWLEFTLENIFWPELKQITWLSVTGLLMVVFGECLRKAAMFTAGSNFNHVVQNEKSDTHTLVTSGVYAWFRHPSYVGWFYWSIGTQVMLCNPICGVSYALTVWRFFRDRTEEEEISLIHFFGEEYLEYKKRVPTGLPFIKGVKVDL",
# DPM1_HUMAN
"MASLEVSRSPRRSRRELEVRSPRQNKYSVLLPTYNERENLPLIVWLLVKSFSESGINYEIIIIDDGSPDGTRDVAEQLEKIYGSDRILLRPREKKLGLGTAYIHGMKHATGNYIIIMDADLSHHPKFIPEFIRKQKEGNFDIVSGTRYKGNGGVYGWDLKRKIISRGANFLTQILLRPGASDLTGSFRLYRKEVLEKLIEKCVSKGYVFQMEMIVRARQLNYTIGEVPISFVDRVYGESKLGGNEIVSFLKGLLTLFATT",
# TMCC2_HUMAN
"MKRCRSDELQQQQGEEDGAGLEDAASHLPGADLRPGETTGANSAGGPTSDAGAAAAPNPGPRSKPPDLKKIQQLSEGSMFGHGLKHLFHSRRRSREREHQTSQDSQQHQQQQGMSDHDSPDEKERSPEMHRVSYAMSLHDLPARPTAFNRVLQQIRSRPSIKRGASLHSSSGGGSSGSSSRRTKSSSLEPQRGSPHLLRKAPQDSSLAAILHQHQCRPRSSSTTDTALLLADGSNVYLLAEEAEGIGDKVDKGDLVALSLPAGHGDTDGPISLDVPDGAPDPQRTKAAIDHLHQKILKITEQIKIEQEARDDNVAEYLKLANNADKQQVSRIKQVFEKKNQKSAQTIAQLHKKLEHYRRRLKEIEQNGPSRQPKDVLRDMQQGLKDVGANVRAGISGFGGGVVEGVKGSLSGLSQATHTAVVSKPREFASLIRNKFGSADNIAHLKDPLEDGPPEEAARALSGSATLVSSPKYGSDDECSSASASSAGAGSNSGAGPGGALGSPKSNALYGAPGNLDALLEELREIKEGQSHLEDSMEDLKTQLQRDYTYMTQCLQEERYRYERLEEQLNDLTELHQNEMTNLKQELASMEEKVAYQSYERARDIQEAVESCLTRVTKLELQQQQQQVVQLEGVENANARALLGKFINVILALMAVLLVFVSTIANFITPLMKTRLRITSTTLLVLVLFLLWKHWDSLTYLLEHVLLPS",
# RTN2_HUMAN
"MGQVLPVFAHCKEAPSTASSTPDSTEGGNDDSDFRELHTAREFSEEDEEETTSQDWGTPRELTFSYIAFDGVVGSGGRRDSTARRPRPQGRSVSEPRDQHPQPSLGDSLESIPSLSQSPEPGRRGDPDTAPPSERPLEDLRLRLDHLGWVARGTGSGEDSSTSSSTPLEDEEPQEPNRLETGEAGEELDLRLRLAQPSSPEVLTPQLSPGSGTPQAGTPSPSRSRDSNSGPEEPLLEEEEKQWGPLEREPVRGQCLDSTDQLEFTVEPRLLGTAMEWLKTSLLLAVYKTVPILELSPPLWTAIGWVQRGPTPPTPVLRVLLKWAKSPRSSGVPSLSLGADMGSKVADLLYWKDTRTSGVVFTGLMVSLLCLLHFSIVSVAAHLALLLLCGTISLRVYRKVLQAVHRGDGANPFQAYLDVDLTLTREQTERLSHQITSRVVSAATQLRHFFLVEDLVDSLKLALLFYILTFVGAIFNGLTLLILGVIGLFTIPLLYRQHQAQIDQYVGLVTNQLSHIKAKIRAKIPGTGALASAAAAVSGSKAKAE",
# PDCD6_HUMAN
"MAAYSYRPGPGAGPGPAAGAALPDQSFLWNVFQRVDKDRSGVISDTELQQALSNGTWTPFNPVTVRSIISMFDRENKAGVNFSEFTGVWKYITDWQNVFRTYDRDNSGMIDKNELKQALSGFGYRLSDQFHDILIRKFDRQGRGQIAFDDFIQGCIVLQRLTDIFRRYDTDQDGWIQVSYEQYLSMVFSIV",
# ERN1_HUMAN
"MPARRLLLLLTLLLPGLGIFGSTSTVTLPETLLFVSTLDGSLHAVSKRTGSIKWTLKEDPVLQVPTHVEEPAFLPDPNDGSLYTLGSKNNEGLTKLPFTIPELVQASPCRSSDGILYMGKKQDIWYVIDLLTGEKQQTLSSAFADSLCPSTSLLYLGRTEYTITMYDTKTRELRWNATYFDYAASLPEDDVDYKMSHFVSNGDGLVVTVDSESGDVLWIQNYASPVVAFYVWQREGLRKVMHINVAVETLRYLTFMSGEVGRITKWKYPFPKETEAKSKLTPTLYVGKYSTSLYASPSMVHEGVAVVPRGSTLPLLEGPQTDGVTIGDKGECVITPSTDVKFDPGLKSKNKLNYLRNYWLLIGHHETPLSASTKMLERFPNNLPKHRENVIPADSEKKSFEEVINLVDQTSENAPTTVSRDVEEKPAHAPARPEAPVDSMLKDMATIILSTFLLIGWVAFIITYPLSMHQQQQLQHQQFQKELEKIQLLQQQQQQLPFHPPGDTAQDGELLDTSGPYSESSGTSSPSTSPRASNHSLCSGSSASKAGSSPSLEQDDGDEETSVVIVGKISFCPKDVLGHGAEGTIVYRGMFDNRDVAVKRILPECFSFADREVQLLRESDEHPNVIRYFCTEKDRQFQYIAIELCAATLQEYVEQKDFAHLGLEPITLLQQTTSGLAHLHSLNIVHRDLKPHNILISMPNAHGKIKAMISDFGLCKKLAVGRHSFSRRSGVPGTEGWIAPEMLSEDCKENPTYTVDIFSAGCVFYYVISEGSHPFGKSLQRQANILLGACSLDCLHPEKHEDVIARELIEKMIAMDPQKRPSAKHVLKHPFFWSLEKQLQFFQDVSDRIEKESLDGPIVKQLERGGRAVVKMDWRENITVPLQTDLRKFRTYKGGSVRDLLRAMRNKKHHYRELPAEVRETLGSLPDDFVCYFTSRFPHLLAHTYRAMELCSHERLFQPYYFHEPPEPQPPVTPDAL",
# ERLN1_HUMAN
"MNMTQARVLVAAVVGLVAVLLYASIHKIEEGHLAVYYRGGALLTSPSGPGYHIMLPFITTFRSVQTTLQTDEVKNVPCGTSGGVMIYIDRIEVVNMLAPYAVFDIVRNYTADYDKTLIFNKIHHELNQFCSAHTLQEVYIELFDQIDENLKQALQKDLNLMAPGLTIQAVRVTKPKIPEAIRRNFELMEAEKTKLLIAAQKQKVVEKEAETERKKAVIEAEKIAQVAKIRFQQKVMEKETEKRISEIEDAAFLAREKAKADAEYYAAHKYATSNKHKLTPEYLELKKYQAIASNSKIYFGSNIPNMFVDSSCALKYSDIRTGRESSLPSKEALEPSGENVIQNKESTG",
# PR15A_HUMAN
"MAPGQAPHQATPWRDAHPFFLLSPVMGLLSRAWSRLRGLGPLEPWLVEAVKGAALVEAGLEGEARTPLAIPHTPWGRRPEEEAEDSGGPGEDRETLGLKTSSSLPEAWGLLDDDDGMYGEREATSVPRGQGSQFADGQRAPLSPSLLIRTLQGSDKNPGEEKAEEEGVAEEEGVNKFSYPPSHRECCPAVEEEDDEEAVKKEAHRTSTSALSPGSKPSTWVSCPGEEENQATEDKRTERSKGARKTSVSPRSSGSDPRSWEYRSGEASEEKEEKAHKETGKGEAAPGPQSSAPAQRPQLKSWWCQPSDEEEGEVKALGAAEKDGEAECPPCIPPPSAFLKAWVYWPGEDTEEEEDEEEDEDSDSGSDEEEGEAEASSSTPATGVFLKSWVYQPGEDTEEEEDEDSDTGSAEDEREAETSASTPPASAFLKAWVYRPGEDTEEEEDEDVDSEDKEDDSEAALGEAESDPHPSHPDQRAHFRGWGYRPGKETEEEEAAEDWGEAEPCPFRVAIYVPGEKPPPPWAPPRLPLRLQRRLKRPETPTHDPDPETPLKARKVRFSEKVTVHFLAVWAGPAQAARQGPWEQLARDRSRFARRITQAQEELSPCLTPAARARAWARLRNPPLAPIPALTQTLPSSSVPSSPVQTTPLSQAVATPSRSSAAAAAALDLSGRRG",
# SOAT2_HUMAN
"MEPGGARLRLQRTEGLGGERERQPCGDGNTETHRAPDLVQWTRHMEAVKAQLLEQAQGQLRELLDRAMREAIQSYPSQDKPLPPPPPGSLSRTQEPSLGKQKVFIIRKSLLDELMEVQHFRTIYHMFIAGLCVFIISTLAIDFIDEGRLLLEFDLLIFSFGQLPLALVTWVPMFLSTLLAPYQALRLWARGTWTQATGLGCALLAAHAVVLCALPVHVAVEHQLPPASRCVLVFEQVRFLMKSYSFLREAVPGTLRARRGEGIQAPSFSSYLYFLFCPTLIYRETYPRTPYVRWNYVAKNFAQALGCVLYACFILGRLCVPVFANMSREPFSTRALVLSILHATLPGIFMLLLIFFAFLHCWLNAFAEMLRFGDRMFYRDWWNSTSFSNYYRTWNVVVHDWLYSYVYQDGLRLLGARARGVAMLGVFLVSAVAHEYIFCFVLGFFYPVMLILFLVIGGMLNFMMHDQRTGPAWNVLMWTMLFLGQGIQVSLYCQEWYARRHCPLPQATFWGLVTPRSWSCHT",
# WFS1_HUMAN
"MDSNTAPLGPSCPQPPPAPQPQARSRLNATASLEQERSERPRAPGPQAGPGPGVRDAAAPAEPQAQHTRSRERADGTGPTKGDMEIPFEEVLERAKAGDPKAQTEVGKHYLQLAGDTDEELNSCTAVDWLVLAAKQGRREAVKLLRRCLADRRGITSENEREVRQLSSETDLERAVRKAALVMYWKLNPKKKKQVAVAELLENVGQVNEHDGGAQPGPVPKSLQKQRRMLERLVSSESKNYIALDDFVEITKKYAKGVIPSSLFLQDDEDDDELAGKSPEDLPLRLKVVKYPLHAIMEIKEYLIDMASRAGMHWLSTIIPTHHINALIFFFIVSNLTIDFFAFFIPLVIFYLSFISMVICTLKVFQDSKAWENFRTLTDLLLRFEPNLDVEQAEVNFGWNHLEPYAHFLLSVFFVIFSFPIASKDCIPCSELAVITGFFTVTSYLSLSTHAEPYTRRALATEVTAGLLSLLPSMPLNWPYLKVLGQTFITVPVGHLVVLNVSVPCLLYVYLLYLFFRMAQLRNFKGTYCYLVPYLVCFMWCELSVVILLESTGLGLLRASIGYFLFLFALPILVAGLALVGVLQFARWFTSLELTKIAVTVAVCSVPLLLRWWTKASFSVVGMVKSLTRSSMVKLILVWLTAIVLFCWFYVYRSEGMKVYNSTLTWQQYGALCGPRAWKETNMARTQILCSHLEGHRVTWTGRFKYVRVTDIDNSAESAINMLPFFIGDWMRCLYGEAYPACSPGNTSTAEEELCRLKLLAKHPCHIKKFDRYKFEITVGMPFSSGADGSRSREEDDVTKDIVLRASSEFKSVLLSLRQGSLIEFSTILEGRLGSKWPVFELKAISCLNCMAQLSPTRRHVKIEHDWRSTVHGAVKFAFDFFFFPFLSAA",
# ERG24_HUMAN
"MAPTQGPRAPLEFGGPLGAAALLLLLPATMFHLLLAARSGPARLLGPPASLPGLEVLWSPRALLLWLAWLGLQAALYLLPARKVAEGQELKDKSRLRYPINGFQALVLTALLVGLGMSAGLPLGALPEMLLPLAFVATLTAFIFSLFLYMKAQVAPVSALAPGGNSGNPIYDFFLGRELNPRICFFDFKYFCELRPGLIGWVLINLALLMKEAELRGSPSLAMWLVNGFQLLYVGDALWHEEAVLTTMDITHDGFGFMLAFGDMAWVPFTYSLQAQFLLHHPQPLGLPMASVICLINATGYYIFRGANSQKNTFRKNPSDPRVAGLETISTATGRKLLVSGWWGMVRHPNYLGDLIMALAWSLPCGVSHLLPYFYLLYFTALLVHREARDERQCLQKYGLAWQEYCRRVPYRIMPYIY",
# AT10B_HUMAN
"MALSVDSSWHRWQWRVRDGFPHCPSETTPLLSPEKGRQSYNLTQQRVVFPNNSIFHQDWEEVSRRYPGNRTCTTKYTLFTFLPRNLFEQFHRWANLYFLFLVILNWMPSMEVFHREITMLPLAIVLFVIMIKDGMEDFKRHRFDKAINCSNIRIYERKEQTYVQKCWKDVRVGDFIQMKCNEIVPADILLLFSSDPNGICHLETASLDGETNLKQRCVVKGFSQQEVQFEPELFHNTIVCEKPNNHLNKFKGYMEHPDQTRTGFGCESLLLRGCTIRNTEMAVGIVIYAGHETKAMLNNSGPRYKRSKIERRMNIDIFFCIGILILMCLIGAVGHSIWNGTFEEHPPFDVPDANGSFLPSALGGFYMFLTMIILLQVLIPISLYVSIELVKLGQVFFLSNDLDLYDEETDLSIQCRALNIAEDLGQIQYIFSDKTGTLTENKMVFRRCTIMGSEYSHQENAKRLETPKELDSDGEEWTQYQCLSFSARWAQDPATMRSQKGAQPLRRSQSARVPIQGHYRQRSMGHRESSQPPVAFSSSIEKDVTPDKNLLTKVRDAALWLETLSDSRPAKASLSTTSSIADFFLALTICNSVMVSTTTEPRQRVTIKPSSKALGTSLEKIQQLFQKLKLLSLSQSFSSTAPSDTDLGESLGANVATTDSDERDDASVCSGGDSTDDGGYRSSMWDQGDILESGSGTSLEEALEAPATDLARPEFCYEAESPDEAALVHAAHAYSFTLVSRTPEQVTVRLPQGTCLTFSLLCTLGFDSVRKRMSVVVRHPLTGEIVVYTKGADSVIMDLLEDPACVPDINMEKKLRKIRARTQKHLDLYARDGLRTLCIAKKVVSEEDFRRWASFRREAEASLDNRDELLMETAQHLENQLTLLGATGIEDRLQEGVPDTIATLREAGIQLWVLTGDKQETAVNIAHSCRLLNQTDTVYTINTENQETCESILNCALEELKQFRELQKPDRKLFGFRLPSKTPSITSEAVVPEAGLVIDGKTLNAIFQGKLEKKFLELTQYCRSVLCCRSTPLQKSMIVKLVRDKLRVMTLSIGDGANDVSMIQAADIGIGISGQEGMQAVMSSDFAITRFKHLKKLLLVHGHWCYSRLARMVVYYLYKNVCYVNLLFWYQFFCGFSSSTMIDYWQMIFFNLFFTSLPPLVFGVLDKDISAETLLALPELYKSGQNSECYNLSTFWISMVDAFYQSLICFFIPYLAYKGSDIDVFTFGTPINTISLTTILLHQAMEMKTWTIFHGVVLLGSFLMYFLVSLLYNATCVICNSPTNPYWVMEGQLSNPTFYLVCFLTPVVALLPRYFFLSLQGTCGKSLISKAQKIDKLPPDKRNLEIQSWRSRQRPAPVPEVARPTHHPVSSITGQDFSASTPKSSNPPKRKHVEESVLHEQRCGTECMRDDSCSGDSSAQLSSGEHLLGPNRIMAYSRGQTDMCRCSKRSSHRRSQSSLTI",
# UFL1_HUMAN
"MADAWEEIRRLAADFQRAQFAEATQRLSERNCIEIVNKLIAQKQLEVVHTLDGKEYITPAQISKEMRDELHVRGGRVNIVDLQQVINVDLIHIENRIGDIIKSEKHVQLVLGQLIDENYLDRLAEEVNDKLQESGQVTISELCKTYDLPGNFLTQALTQRLGRIISGHIDLDNRGVIFTEAFVARHKARIRGLFSAITRPTAVNSLISKYGFQEQLLYSVLEELVNSGRLRGTVVGGRQDKAVFVPDIYSRTQSTWVDSFFRQNGYLEFDALSRLGIPDAVSYIKKRYKTTQLLFLKAACVGQGLVDQVEASVEEAISSGTWVDIAPLLPTSLSVEDAAILLQQVMRAFSKQASTVVFSDTVVVSEKFINDCTELFRELMHQKAEKEMKNNPVHLITEEDLKQISTLESVSTSKKDKKDERRRKATEGSGSMRGGGGGNAREYKIKKVKKKGRKDDDSDDESQSSHTGKKKPEISFMFQDEIEDFLRKHIQDAPEEFISELAEYLIKPLNKTYLEVVRSVFMSSTTSASGTGRKRTIKDLQEEVSNLYNNIRLFEKGMKFFADDTQAALTKHLLKSVCTDITNLIFNFLASDLMMAVDDPAAITSEIRKKILSKLSEETKVALTKLHNSLNEKSIEDFISCLDSAAEACDIMVKRGDKKRERQILFQHRQALAEQLKVTEDPALILHLTSVLLFQFSTHSMLHAPGRCVPQIIAFLNSKIPEDQHALLVKYQGLVVKQLVSQSKKTGQGDYPLNNELDKEQEDVASTTRKELQELSSSIKDLVLKSRKSSVTEE",
# ERLN2_HUMAN
"MAQLGAVVAVASSFFCASLFSAVHKIEEGHIGVYYRGGALLTSTSGPGFHLMLPFITSYKSVQTTLQTDEVKNVPCGTSGGVMIYFDRIEVVNFLVPNAVYDIVKNYTADYDKALIFNKIHHELNQFCSVHTLQEVYIELFDQIDENLKLALQQDLTSMAPGLVIQAVRVTKPNIPEAIRRNYELMESEKTKLLIAAQKQKVVEKEAETERKKALIEAEKVAQVAEITYGQKVMEKETEKKISEIEDAAFLAREKAKADAECYTAMKIAEANKLKLTPEYLQLMKYKAIASNSKIYFGKDIPNMFMDSAGSVSKQFEGLADKLSFGLEDEPLETATKEN",
# SC31A_HUMAN
"MKLKEVDRTAMQAWSPAQNHPIYLATGTSAQQLDATFSTNASLEIFELDLSDPSLDMKSCATFSSSHRYHKLIWGPYKMDSKGDVSGVLIAGGENGNIILYDPSKIIAGDKEVVIAQNDKHTGPVRALDVNIFQTNLVASGANESEIYIWDLNNFATPMTPGAKTQPPEDISCIAWNRQVQHILASASPSGRATVWDLRKNEPIIKVSDHSNRMHCSGLAWHPDVATQMVLASEDDRLPVIQMWDLRFASSPLRVLENHARGILAIAWSMADPELLLSCGKDAKILCSNPNTGEVLYELPTNTQWCFDIQWCPRNPAVLSAASFDGRISVYSIMGGSTDGLRQKQVDKLSSSFGNLDPFGTGQPLPPLQIPQQTAQHSIVLPLKKPPKWIRRPVGASFSFGGKLVTFENVRMPSHQGAEQQQQQHHVFISQVVTEKEFLSRSDQLQQAVQSQGFINYCQKKIDASQTEFEKNVWSFLKVNFEDDSRGKYLELLGYRKEDLGKKIALALNKVDGANVALKDSDQVAQSDGEESPAAEEQLLGEHIKEEKEESEFLPSSGGTFNISVSGDIDGLITQALLTGNFESAVDLCLHDNRMADAIILAIAGGQELLARTQKKYFAKSQSKITRLITAVVMKNWKEIVESCDLKNWREALAAVLTYAKPDEFSALCDLLGTRLENEGDSLLQTQACLCYICAGNVEKLVACWTKAQDGSHPLSLQDLIEKVVILRKAVQLTQAMDTSTVGVLLAAKMSQYANLLAAQGSIAAALAFLPDNTNQPNIMQLRDRLCRAQGEPVAGHESPKIPYEKQQLPKGRPGPVAGHHQMPRVQTQQYYPHGENPPPPGFIMHGNVNPNAAGQLPTSPGHMHTQVPPYPQPQPYQPAQPYPFGTGGSAMYRPQQPVAPPTSNAYPNTPYISSASSYTGQSQLYAAQHQASSPTSSPATSFPPPPSSGASFQHGGPGAPPSSSAYALPPGTTGTLPAASELPASQRTGPQNGWNDPPALNRVPKKKKMPENFMPPVPITSPIMNPLGDPQSQMLQQQPSAPVPLSSQSSFPQPHLPGGQPFHGVQQPLGQTGMPPSFSKPNIEGAPGAPIGNTFQHVQSLPTKKITKKPIPDEHLILKTTFEDLIQRCLSSATDPQTKRKLDDASKRLEFLYDKLREQTLSPTITSGLHNIARSIETRNYSEGLTMHTHIVSTSNFSETSAFMPVLKVVLTQANKLGV",
# RTN3_HUMAN
"MAEPSAATQSHSISSSSFGAEPSAPGGGGSPGACPALGTKSCSSSCADSFVSSSSSQPVSLFSTSQEGLSSLCSDEPSSEIMTSSFLSSSEIHNTGLTILHGEKSHVLGSQPILAKEGKDHLDLLDMKKMEKPQGTSNNVSDSSVSLAAGVHCDRPSIPASFPEHPAFLSKKIGQVEEQIDKETKNPNGVSSREAKTALDADDRFTLLTAQKPPTEYSKVEGIYTYSLSPSKVSGDDVIEKDSPESPFEVIIDKAAFDKEFKDSYKESTDDFGSWSVHTDKESSEDISETNDKLFPLRNKEAGRYPMSALLSRQFSHTNAALEEVSRCVNDMHNFTNEILTWDLVPQVKQQTDKSSDCITKTTGLDMSEYNSEIPVVNLKTSTHQKTPVCSIDGSTPITKSTGDWAEASLQQENAITGKPVPDSLNSTKEFSIKGVQGNMQKQDDTLAELPGSPPEKCDSLGSGVATVKVVLPDDHLKDEMDWQSSALGEITEADSSGESDDTVIEDITADTSFENNKIQAEKPVSIPSAVVKTGEREIKEIPSCEREEKTSKNFEELVSDSELHQDQPDILGRSPASEAACSKVPDTNVSLEDVSEVAPEKPITTENPKLPSTVSPNVFNETEFSLNVTTSAYLESLHGKNVKHIDDSSPEDLIAAFTETRDKGIVDSERNAFKAISEKMTDFKTTPPVEVLHENESGGSEIKDIGSKYSEQSKETNGSEPLGVFPTQGTPVASLDLEQEQLTIKALKELGERQVEKSTSAQRDAELPSEEVLKQTFTFAPESWPQRSYDILERNVKNGSDLGISQKPITIRETTRVDAVSSLSKTELVKKHVLARLLTDFSVHDLIFWRDVKKTGFVFGTTLIMLLSLAAFSVISVVSYLILALLSVTISFRIYKSVIQAVQKSEEGHPFKAYLDVDITLSSEAFHNYMNAAMVHINRALKLIIRLFLVEDLVDSLKLAVFMWLMTYVGAVFNGITLLILAELLIFSVPIVYEKYKTQIDHYVGIARDQTKSIVEKIQAKLPGIAKKKAE",
# STBD1_HUMAN
"MGAVWSALLVGGGLAGALFVWLLRGGPGDTGKDGDAEQEKDAPLGGAAIPGGHQSGSSGLSPGPSGQELVTKPEHLQESNGHLISKTKDLGKLQAASWRLQNPSREVCDNSREHVPSGQFPDTEAPATSETSNSRSYSEVSRNESLESPMGEWGFQKGQEISAKAATCFAEKLPSSNLLKNRAKEEMSLSDLNSQDRVDHEEWEMVPRHSSWGDVGVGGSLKAPVLNLNQGMDNGRSTLVEARGQQVHGKMERVAVMPAGSQQVSVRFQVHYVTSTDVQFIAVTGDHECLGRWNTYIPLHYNKDGFWSHSIFLPADTVVEWKFVLVENGGVTRWEECSNRFLETGHEDKVVHAWWGIH",
# VAPB_HUMAN
"MAKVEQVLSLEPQHELKFRGPFTDVVTTNLKLGNPTDRNVCFKVKTTAPRRYCVRPNSGIIDAGASINVSVMLQPFDYDPNEKSKHKFMVQSMFAPTDTSDMEAVWKEAKPEDLMDSKLRCVFELPAENDKPHDVEINKIISTTASKTETPIVSKSLSSSLDDTEVKKVMEECKRLQGEVQRLREENKQFKEEDGLRMRKTVQSNSPISALAPTGKEEGLSTRLLALVVLFFIVGVIIGKIAL",
# SGPL1_HUMAN
"MPSTDLLMLKAFEPYLEILEVYSTKAKNYVNGHCTKYEPWQLIAWSVVWTLLIVWGYEFVFQPESLWSRFKKKCFKLTRKMPIIGRKIQDKLNKTKDDISKNMSFLKVDKEYVKALPSQGLSSSAVLEKLKEYSSMDAFWQEGRASGTVYSGEEKLTELLVKAYGDFAWSNPLHPDIFPGLRKIEAEIVRIACSLFNGGPDSCGCVTSGGTESILMACKAYRDLAFEKGIKTPEIVAPQSAHAAFNKAASYFGMKIVRVPLTKMMEVDVRAMRRAISRNTAMLVCSTPQFPHGVIDPVPEVAKLAVKYKIPLHVDACLGGFLIVFMEKAGYPLEHPFDFRVKGVTSISADTHKYGYAPKGSSLVLYSDKKYRNYQFFVDTDWQGGIYASPTIAGSRPGGISAACWAALMHFGENGYVEATKQIIKTARFLKSELENIKGIFVFGNPQLSVIALGSRDFDIYRLSNLMTAKGWNLNQLQFPPSIHFCITLLHARKRVAIQFLKDIRESVTQIMKNPKAKTTGMGAIYGMAQTTVDRNMVAELSSVFLDSLYSTDTVTQGSQMNGSPKPH",
# ACSL3_HUMAN
"MNNHVSSKPSTMKLKHTINPILLYFIHFLISLYTILTYIPFYFFSESRQEKSNRIKAKPVNSKPDSAYRSVNSLDGLASVLYPGCDTLDKVFTYAKNKFKNKRLLGTREVLNEEDEVQPNGKIFKKVILGQYNWLSYEDVFVRAFNFGNGLQMLGQKPKTNIAIFCETRAEWMIAAQACFMYNFQLVTLYATLGGPAIVHALNETEVTNIITSKELLQTKLKDIVSLVPRLRHIITVDGKPPTWSEFPKGIIVHTMAAVEALGAKASMENQPHSKPLPSDIAVIMYTSGSTGLPKGVMISHSNIIAGITGMAERIPELGEEDVYIGYLPLAHVLELSAELVCLSHGCRIGYSSPQTLADQSSKIKKGSKGDTSMLKPTLMAAVPEIMDRIYKNVMNKVSEMSSFQRNLFILAYNYKMEQISKGRNTPLCDSFVFRKVRSLLGGNIRLLLCGGAPLSATTQRFMNICFCCPVGQGYGLTESAGAGTISEVWDYNTGRVGAPLVCCEIKLKNWEEGGYFNTDKPHPRGEILIGGQSVTMGYYKNEAKTKADFFEDENGQRWLCTGDIGEFEPDGCLKIIDRKKDLVKLQAGEYVSLGKVEAALKNLPLVDNICAYANSYHSYVIGFVVPNQKELTELARKKGLKGTWEELCNSCEMENEVLKVLSEAAISASLEKFEIPVKIRLSPEPWTPETGLVTDAFKLKRKELKTHYQADIERMYGRK",
# FTCD_HUMAN
"MSQLVECVPNFSEGKNQEVIDAISGAITQTPGCVLLDVDAGPSTNRTVYTFVGPPECVVEGALNAARVASRLIDMSRHQGEHPRMGALDVCPFIPVRGVSVDECVLCAQAFGQRLAEELDVPVYLYGEAARMDSRRTLPAIRAGEYEALPKKLQQADWAPDFGPSSFVPSWGATATGARKFLIAFNINLLGTKEQAHRIALNLREQGRGKDQPGRLKKVQGIGWYLDEKNLAQVSTNLLDFEVTALHTVYEETCREAQELSLPVVGSQLVGLVPLKALLDAAAFYCEKENLFILEEEQRIRLVVSRLGLDSLCPFSPKERIIEYLVPERGPERGLGSKSLRAFVGEVGARSAAPGGGSVAAAAAAMGAALGSMVGLMTYGRRQFQSLDTTMRRLIPPFREASAKLTTLVDADAEAFTAYLEAMRLPKNTPEEKDRRTAALQEGLRRAVSVPLTLAETVASLWPALQELARCGNLACRSDLQVAAKALEMGVFGAYFNVLINLRDITDEAFKDQIHHRVSSLLQEAKTQAALVLDCLETRQE",
# NB5R3_HUMAN
"MGAQLSTLGHMVLFPVWFLYSLLMKLFQRSTPAITLESPDIKYPLRLIDREIISHDTRRFRFALPSPQHILGLPVGQHIYLSARIDGNLVVRPYTPISSDDDKGFVDLVIKVYFKDTHPKFPAGGKMSQYLESMQIGDTIEFRGPSGLLVYQGKGKFAIRPDKKSNPIIRTVKSVGMIAGGTGITPMLQVIRAIMKDPDDHTVCHLLFANQTEKDILLRPELEELRNKHSARFKLWYTLDRAPEAWDYGQGFVNEEMIRDHLPPPEEEPLVLMCGPPPMIQYACLPNLDHVGHPTERCFVF",
# HMDH_HUMAN
"MLSRLFRMHGLFVASHPWEVIVGTVTLTICMMSMNMFTGNNKICGWNYECPKFEEDVLSSDIIILTITRCIAILYIYFQFQNLRQLGSKYILGIAGLFTIFSSFVFSTVVIHFLDKELTGLNEALPFFLLLIDLSRASTLAKFALSSNSQDEVRENIARGMAILGPTFTLDALVECLVIGVGTMSGVRQLEIMCCFGCMSVLANYFVFMTFFPACVSLVLELSRESREGRPIWQLSHFARVLEEEENKPNPVTQRVKMIMSLGLVLVHAHSRWIADPSPQNSTADTSKVSLGLDENVSKRIEPSVSLWQFYLSKMISMDIEQVITLSLALLLAVKYIFFEQTETESTLSLKNPITSPVVTQKKVPDNCCRREPMLVRNNQKCDSVEEETGINRERKVEVIKPLVAETDTPNRATFVVGNSSLLDTSSVLVTQEPEIELPREPRPNEECLQILGNAEKGAKFLSDAEIIQLVNAKHIPAYKLETLMETHERGVSIRRQLLSKKLSEPSSLQYLPYRDYNYSLVMGACCENVIGYMPIPVGVAGPLCLDEKEFQVPMATTEGCLVASTNRGCRAIGLGGGASSRVLADGMTRGPVVRLPRACDSAEVKAWLETSEGFAVIKEAFDSTSRFARLQKLHTSIAGRNLYIRFQSRSGDAMGMNMISKGTEKALSKLHEYFPEMQILAVSGNYCTDKKPAAINWIEGRGKSVVCEAVIPAKVVREVLKTTTEAMIEVNINKNLVGSAMAGSIGGYNAHAANIVTAIYIACGQDAAQNVGSSNCITLMEASGPTNEDLYISCTMPSIEIGTVGGGTNLLPQQACLQMLGVQGACKDNPGENARQLARIVCGTVMAGELSLMAALAAGHLVKSHMIHNRSKINLQDLQGACTKKTA",
# RPN1_HUMAN
"MEAPAAGLFLLLLLGTWAPAPGSASSEAPPLINEDVKRTVDLSSHLAKVTAEVVLAHLGGGSTSRATSFLLALEPELEARLAHLGVQVKGEDEEENNLEVRETKIKGKSGRFFTVKLPVALDPGAKISVIVETVYTHVLHPYPTQITQSEKQFVVFEGNHYFYSPYPTKTQTMRVKLASRNVESYTKLGNPTRSEDLLDYGPFRDVPAYSQDTFKVHYENNSPFLTITSMTRVIEVSHWGNIAVEENVDLKHTGAVLKGPFSRYDYQRQPDSGISSIRSFKTILPAAAQDVYYRDEIGNVSTSHLLILDDSVEMEIRPRFPLFGGWKTHYIVGYNLPSYEYLYNLGDQYALKMRFVDHVFDEQVIDSLTVKIILPEGAKNIEIDSPYEISRAPDELHYTYLDTFGRPVIVAYKKNLVEQHIQDIVVHYTFNKVLMLQEPLLVVAAFYILFFTVIIYVRLDFSITKDPAAEARMKVACITEQVLTLVNKRIGLYRHFDETVNRYKQSRDISTLNSGKKSLETEHKALTSEIALLQSRLKTEGSDLCDRVSEMQKLDAQVKELVLKSAVEAERLVAGKLKKDTYIENEKLISGKRQELVTKIDHILDAL",
# CP17A_HUMAN
"MWELVALLLLTLAYLFWPKRRCPGAKYPKSLLSLPLVGSLPFLPRHGHMHNNFFKLQKKYGPIYSVRMGTKTTVIVGHHQLAKEVLIKKGKDFSGRPQMATLDIASNNRKGIAFADSGAHWQLHRRLAMATFALFKDGDQKLEKIICQEISTLCDMLATHNGQSIDISFPVFVAVTNVISLICFNTSYKNGDPELNVIQNYNEGIIDNLSKDSLVDLVPWLKIFPNKTLEKLKSHVKIRNDLLNKILENYKEKFRSDSITNMLDTLMQAKMNSDNGNAGPDQDSELLSDNHILTTIGDIFGAGVETTTSVVKWTLAFLLHNPQVKKKLYEEIDQNVGFSRTPTISDRNRLLLLEATIREVLRLRPVAPMLIPHKANVDSSIGEFAVDKGTEVIINLWALHHNEKEWHQPDQFMPERFLNPAGTQLISPSVSYLPFGAGPRSCIGEILARQELFLIMAWLLQRFDLEVPDDGQLPSLEGIPKVVFLIDSFKVKIKVRQAWREAQAEGST",
# SRPRA_HUMAN
"MLDFFTIFSKGGLVLWCFQGVSDSCTGPVNALIRSVLLQERGGNNSFTHEALTLKYKLDNQFELVFVVGFQKILTLTYVDKLIDDVHRLFRDKYRTEIQQQSALSLLNGTFDFQNDFLRLLREAEESSKIRAPTTMKKFEDSEKAKKPVRSMIETRGEKPKEKAKNSKKKGAKKEGSDGPLATSKPVPAEKSGLPVGPENGVELSKEELIRRKREEFIQKHGRGMEKSNKSTKSDAPKEKGKKAPRVWELGGCANKEVLDYSTPTTNGTPEAALSEDINLIRGTGSGGQLQDLDCSSSDDEGAAQNSTKPSATKGTLGGMFGMLKGLVGSKSLSREDMESVLDKMRDHLIAKNVAADIAVQLCESVANKLEGKVMGTFSTVTSTVKQALQESLVQILQPQRRVDMLRDIMDAQRRQRPYVVTFCGVNGVGKSTNLAKISFWLLENGFSVLIAACDTFRAGAVEQLRTHTRRLSALHPPEKHGGRTMVQLFEKGYGKDAAGIAMEAIAFARNQGFDVVLVDTAGRMQDNAPLMTALAKLITVNTPDLVLFVGEALVGNEAVDQLVKFNRALADHSMAQTPRLIDGIVLTKFDTIDDKVGAAISMTYITSKPIVFVGTGQTYCDLRSLNAKAVVAALMKA",
# STS_HUMAN
"MPLRKMKIPFLLLFFLWEAESHAASRPNIILVMADDLGIGDPGCYGNKTIRTPNIDRLASGGVKLTQHLAASPLCTPSRAAFMTGRYPVRSGMASWSRTGVFLFTASSGGLPTDEITFAKLLKDQGYSTALIGKWHLGMSCHSKTDFCHHPLHHGFNYFYGISLTNLRDCKPGEGSVFTTGFKRLVFLPLQIVGVTLLTLAALNCLGLLHVPLGVFFSLLFLAALILTLFLGFLHYFRPLNCFMMRNYEIIQQPMSYDNLTQRLTVEAAQFIQRNTETPFLLVLSYLHVHTALFSSKDFAGKSQHGVYGDAVEEMDWSVGQILNLLDELRLANDTLIYFTSDQGAHVEEVSSKGEIHGGSNGIYKGGKANNWEGGIRVPGILRWPRVIQAGQKIDEPTSNMDIFPTVAKLAGAPLPEDRIIDGRDLMPLLEGKSQRSDHEFLFHYCNAYLNAVRWHPQNSTSIWKAFFFTPNFNPVGSNGCFATHVCFCFGSYVTHHDPPLLFDISKDPRERNPLTPASEPRFYEILKVMQEAADRHTQTLPEVPDQFSWNNFLWKPWLQLCCPSTGLSCQCDREKQDKRLSR",
# HMOX1_HUMAN
"MERPQPDSMPQDLSEALKEATKEVHTQAENAEFMRNFQKGQVTRDGFKLVMASLYHIYVALEEEIERNKESPVFAPVYFPEELHRKAALEQDLAFWYGPRWQEVIPYTPAMQRYVKRLHEVGRTEPELLVAHAYTRYLGDLSGGQVLKKIAQKALDLPSSGEGLAFFTFPNIASATKFKQLYRSRMNSLEMTPAVRQRVIEEAKTAFLLNIQLFEELQELLTHDTKDQSPSRAPGLRQRASNKVQDSAPVETPRGKPPLNTRSQAPLLRWVLTLSFLVATVAVGLYAM",
# BCL2_HUMAN
"MAHAGRTGYDNREIVMKYIHYKLSQRGYEWDAGDVGAAPPGAAPAPGIFSSQPGHTPHPAASRDPVARTSPLQTPAAPGAAAGPALSPVPPVVHLTLRQAGDDFSRRYRRDFAEMSSQLHLTPFTARGRFATVVEELFRDGVNWGRIVAFFEFGGVMCVESVNREMSPLVDNIALWMTEYLNRHLHTWIQDNGGWDAFVELYGPSMRPLFDFSWLSLKTLLSLALVGACITLGAYLGHK",
# MGST1_HUMAN
"MVDLTQVMDDEVFMAFASYATIILSKMMLMSTATAFYRLTRKVFANPEDCVAFGKGENAKKYLRTDDRVERVRRAHLNDLENIIPFLGIGLLYSLSGPDPSTAILHFRLFVGARIYHTIAYLTPLPQPNRALSFFVGYGVTLSMAYRLLKSKLYL",
# CP2D6_HUMAN
"MGLEALVPLAVIVAIFLLLVDLMHRRQRWAARYPPGPLPLPGLGNLLHVDFQNTPYCFDQLRRRFGDVFSLQLAWTPVVVLNGLAAVREALVTHGEDTADRPPVPITQILGFGPRSQGVFLARYGPAWREQRRFSVSTLRNLGLGKKSLEQWVTEEAACLCAAFANHSGRPFRPNGLLDKAVSNVIASLTCGRRFEYDDPRFLRLLDLAQEGLKEESGFLREVLNAVPVLLHIPALAGKVLRFQKAFLTQLDELLTEHRMTWDPAQPPRDLTEAFLAEMEKAKGNPESSFNDENLRIVVADLFSAGMVTTSTTLAWGLLLMILHPDVQRRVQQEIDDVIGQVRRPEMGDQAHMPYTTAVIHEVQRFGDIVPLGVTHMTSRDIEVQGFRIPKGTTLITNLSSVLKDEAVWEKPFRFHPEHFLDAQGHFVKPEAFLPFSAGRRACLGEPLARMELFLFFTSLLQHFSFSVPTGQPRPSHHGVFAFLVSPSPYELCAVPR",
# BIP_HUMAN
"MKLSLVAAMLLLLSAARAEEEDKKEDVGTVVGIDLGTTYSCVGVFKNGRVEIIANDQGNRITPSYVAFTPEGERLIGDAAKNQLTSNPENTVFDAKRLIGRTWNDPSVQQDIKFLPFKVVEKKTKPYIQVDIGGGQTKTFAPEEISAMVLTKMKETAEAYLGKKVTHAVVTVPAYFNDAQRQATKDAGTIAGLNVMRIINEPTAAAIAYGLDKREGEKNILVFDLGGGTFDVSLLTIDNGVFEVVATNGDTHLGGEDFDQRVMEHFIKLYKKKTGKDVRKDNRAVQKLRREVEKAKRALSSQHQARIEIESFYEGEDFSETLTRAKFEELNMDLFRSTMKPVQKVLEDSDLKKSDIDEIVLVGGSTRIPKIQQLVKEFFNGKEPSRGINPDEAVAYGAAVQAGVLSGDQDTGDLVLLDVCPLTLGIETVGGVMTKLIPRNTVVPTKKSQIFSTASDNQPTVTIKVYEGERPLTKDNHLLGTFDLTGIPPAPRGVPQIEVTFEIDVNGILRVTAEDKGTGNKNKITITNDQNRLTPEEIERMVNDAEKFAEEDKKLKERIDTRNELESYAYSLKNQIGDKEKLGGKLSSEDKETMEKAVEEKIEWLESHQDADIEDFKAKKKELEEIVQPIISKLYGSAGPPPTGEEDTAEKDEL",
# CP19A_HUMAN
"MVLEMLNPIHYNITSIVPEAMPAATMPVLLLTGLFLLVWNYEGTSSIPGPGYCMGIGPLISHGRFLWMGIGSACNYYNRVYGEFMRVWISGEETLIISKSSSMFHIMKHNHYSSRFGSKLGLQCIGMHEKGIIFNNNPELWKTTRPFFMKALSGPGLVRMVTVCAESLKTHLDRLEEVTNESGYVDVLTLLRRVMLDTSNTLFLRIPLDESAIVVKIQGYFDAWQALLIKPDIFFKISWLYKKYEKSVKDLKDAIEVLIAEKRRRISTEEKLEECMDFATELILAEKRGDLTRENVNQCILEMLIAAPDTMSVSLFFMLFLIAKHPNVEEAIIKEIQTVIGERDIKIDDIQKLKVMENFIYESMRYQPVVDLVMRKALEDDVIDGYPVKKGTNIILNIGRMHRLEFFPKPNEFTLENFAKNVPYRYFQPFGFGPRGCAGKYIAMVMMKAILVTLLRRFHVKTLQGQCVESIQKIHDLSLHPDETKNMLEMIFTPRNSDRCLEH",
# PA2GA_HUMAN
"MKTLLLLAVIMIFGLLQAHGNLVNFHRMIKLTTGKEAALSYGFYGCHCGVGGRGSPKDATDRCCVTHDCCYKRLEKRGCGTKFLSYKFSNSGSRITCAKQDSCRSQLCECDKAAATCFARNKTTYNKKYQYYSNKHCRGSTPRC",
# ENPL_HUMAN
"MRALWVLGLCCVLLTFGSVRADDEVDVDGTVEEDLGKSREGSRTDDEVVQREEEAIQLDGLNASQIRELREKSEKFAFQAEVNRMMKLIINSLYKNKEIFLRELISNASDALDKIRLISLTDENALSGNEELTVKIKCDKEKNLLHVTDTGVGMTREELVKNLGTIAKSGTSEFLNKMTEAQEDGQSTSELIGQFGVGFYSAFLVADKVIVTSKHNNDTQHIWESDSNEFSVIADPRGNTLGRGTTITLVLKEEASDYLELDTIKNLVKKYSQFINFPIYVWSSKTETVEEPMEEEEAAKEEKEESDDEAAVEEEEEEKKPKTKKVEKTVWDWELMNDIKPIWQRPSKEVEEDEYKAFYKSFSKESDDPMAYIHFTAEGEVTFKSILFVPTSAPRGLFDEYGSKKSDYIKLYVRRVFITDDFHDMMPKYLNFVKGVVDSDDLPLNVSRETLQQHKLLKVIRKKLVRKTLDMIKKIADDKYNDTFWKEFGTNIKLGVIEDHSNRTRLAKLLRFQSSHHPTDITSLDQYVERMKEKQDKIYFMAGSSRKEAESSPFVERLLKKGYEVIYLTEPVDEYCIQALPEFDGKRFQNVAKEGVKFDESEKTKESREAVEKEFEPLLNWMKDKALKDKIEKAVVSQRLTESPCALVASQYGWSGNMERIMKAQAYQTGKDISTNYYASQKKTFEINPRHPLIRDMLRRIKEDEDDKTVLDLAVVLFETATLRSGYLLPDTKAYGDRIERMLRLSLNIDPDAKVEEEPEEEPEETAEDTTEDTEQDEDEEMDVGTDEEEETAKESTAEKDEL",
# CD1D_HUMAN
"MGCLLFLLLWALLQAWGSAEVPQRLFPLRCLQISSFANSSWTRTDGLAWLGELQTHSWSNDSDTVRSLKPWSQGTFSDQQWETLQHIFRVYRSSFTRDVKEFAKMLRLSYPLELQVSAGCEVHPGNASNNFFHVAFQGKDILSFQGTSWEPTQEAPLWVNLAIQVLNQDKWTRETVQWLLNGTCPQFVSGLLESGKSELKKQVKPKAWLSRGPSPGPGRLLLVCHVSGFYPKPVWVKWMRGEQEQQGTQPGDILPNADETWYLRATLDVVAGEAAGLSCRVKHSSLEGQDIVLYWGGSYTSMGLIALAVLACLLFLLIVGFTSRFKRQTSYQGVL",
# AT2A2_HUMAN
"MENAHTKTVEEVLGHFGVNESTGLSLEQVKKLKERWGSNELPAEEGKTLLELVIEQFEDLLVRILLLAACISFVLAWFEEGEETITAFVEPFVILLILVANAIVGVWQERNAENAIEALKEYEPEMGKVYRQDRKSVQRIKAKDIVPGDIVEIAVGDKVPADIRLTSIKSTTLRVDQSILTGESVSVIKHTDPVPDPRAVNQDKKNMLFSGTNIAAGKAMGVVVATGVNTEIGKIRDEMVATEQERTPLQQKLDEFGEQLSKVISLICIAVWIINIGHFNDPVHGGSWIRGAIYYFKIAVALAVAAIPEGLPAVITTCLALGTRRMAKKNAIVRSLPSVETLGCTSVICSDKTGTLTTNQMSVCRMFILDRVEGDTCSLNEFTITGSTYAPIGEVHKDDKPVNCHQYDGLVELATICALCNDSALDYNEAKGVYEKVGEATETALTCLVEKMNVFDTELKGLSKIERANACNSVIKQLMKKEFTLEFSRDRKSMSVYCTPNKPSRTSMSKMFVKGAPEGVIDRCTHIRVGSTKVPMTSGVKQKIMSVIREWGSGSDTLRCLALATHDNPLRREEMHLEDSANFIKYETNLTFVGCVGMLDPPRIEVASSVKLCRQAGIRVIMITGDNKGTAVAICRRIGIFGQDEDVTSKAFTGREFDELNPSAQRDACLNARCFARVEPSHKSKIVEFLQSFDEITAMTGDGVNDAPALKKAEIGIAMGSGTAVAKTASEMVLADDNFSTIVAAVEEGRAIYNNMKQFIRYLISSNVGEVVCIFLTAALGFPEALIPVQLLWVNLVTDGLPATALGFNPPDLDIMNKPPRNPKEPLISGWLFFRYLAIGCYVGAATVGAAAWWFIAADGGPRVSFYQLSHFLQCKEDNPDFEGVDCAIFESPYPMTMALSVLVTIEMCNALNSLSENQSLLRMPPWENIWLVGSICLSMSLHFLILYVEPLPLIFQITPLNVTQWLMVLKISLPVILMDETLKFVARNYLEPGKECVQPATKSCSFSACTDGISWPFVLLIMPLVIWVYSTDTNFSDMFWS",
# XBP1_HUMAN
"MVVVAAAPNPADGTPKVLLLSGQPASAAGAPAGQALPLMVPAQRGASPEAASGGLPQARKRQRLTHLSPEEKALRRKLKNRVAAQTARDRKKARMSELEQQVVDLEEENQKLLLENQLLREKTHGLVVENQELRQRLGMDALVAEEEAEAKGNEVRPVAGSAESAALRLRAPLQQVQAQLSPLQNISPWILAVLTLQIQSLISCWAFWTTWTQSCSSNALPQSLPAWRSSQRSTQKDPVPYQPPFLCQWGRHQPSWKPLMN",
# PTN1_HUMAN
"MEMEKEFEQIDKSGSWAAIYQDIRHEASDFPCRVAKLPKNKNRNRYRDVSPFDHSRIKLHQEDNDYINASLIKMEEAQRSYILTQGPLPNTCGHFWEMVWEQKSRGVVMLNRVMEKGSLKCAQYWPQKEEKEMIFEDTNLKLTLISEDIKSYYTVRQLELENLTTQETREILHFHYTTWPDFGVPESPASFLNFLFKVRESGSLSPEHGPVVVHCSAGIGRSGTFCLADTCLLLMDKRKDPSSVDIKKVLLEMRKFRMGLIQTADQLRFSYLAVIEGAKFIMGDSSVQDQWKELSHEDLEPPPEHIPPPPRPPKRILEPHNGKCREFFPNHQWVKEETQEDKDCPIKEEKGSPLNAAPYGIESMSQDTEVRSRVVGGSLRGAQAASPAKGEPSLPEKDEDHALSYWKPFLVNMCVATVLTAGAYLCYRFLFNSNT",
# ATF6A_HUMAN
"MGEPAGVAGTMESPFSPGLFHRLDEDWDSALFAELGYFTDTDELQLEAANETYENNFDNLDFDLDLMPWESDIWDINNQICTVKDIKAEPQPLSPASSSYSVSSPRSVDSYSSTQHVPEELDLSSSSQMSPLSLYGENSNSLSSAEPLKEDKPVTGPRNKTENGLTPKKKIQVNSKPSIQPKPLLLPAAPKTQTNSSVPAKTIIIQTVPTLMPLAKQQPIISLQPAPTKGQTVLLSQPTVVQLQAPGVLPSAQPVLAVAGGVTQLPNHVVNVVPAPSANSPVNGKLSVTKPVLQSTMRNVGSDIAVLRRQQRMIKNRESACQSRKKKKEYMLGLEARLKAALSENEQLKKENGTLKRQLDEVVSENQRLKVPSPKRRVVCVMIVLAFIILNYGPMSMLEQDSRRMNPSVSPANQRRHLLGFSAKEAQDTSDGIIQKNSYRYDHSVSNDKALMVLTEEPLLYIPPPPCQPLINTTESLRLNHELRGWVHRHEVERTKSRRMTNNQQKTRILQGALEQGSNSQLMAVQYTETTSSISRNSGSELQVYYASPRSYQDFFEAIRRRGDTFYVVSFRRDHLLLPATTHNKTTRPKMSIVLPAININENVINGQDYEVMMQIDCQVMDTRILHIKSSSVPPYLRDQQRNQTNTFFGSPPAATEATHVVSTIPESLQ",
# UD16_HUMAN
"MACLLRSFQRISAGVFFLALWGMVVGDKLLVVPQDGSHWLSMKDIVEVLSDRGHEIVVVVPEVNLLLKESKYYTRKIYPVPYDQEELKNRYQSFGNNHFAERSFLTAPQTEYRNNMIVIGLYFINCQSLLQDRDTLNFFKESKFDALFTDPALPCGVILAEYLGLPSVYLFRGFPCSLEHTFSRSPDPVSYIPRCYTKFSDHMTFSQRVANFLVNLLEPYLFYCLFSKYEELASAVLKRDVDIITLYQKVSVWLLRYDFVLEYPRPVMPNMVFIGGINCKKRKDLSQEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# AL5AP_HUMAN
"MDQETVGNVVLLAIVTLISVVQNGFFAHKVEHESRTQNGRSFQRTGTLAFERVYTANQNCVDAYPTFLAVLWSAGLLCSQVPAAFAGLMYLFVRQKYFVGYLGERTQSTPGYIFGKRIILFLFLMSVAGIFNYYLIFFFGSDFENYIKTISTTISPLLLIP",
# UD11_HUMAN
"MAVESQGGRPLVLGLLLCVLGPVVSHAGKILLIPVDGSHWLSMLGAIQQLQQRGHEIVVLAPDASLYIRDGAFYTLKTYPVPFQREDVKESFVSLGHNVFENDSFLQRVIKTYKKIKKDSAMLLSGCSHLLHNKELMASLAESSFDVMLTDPFLPCSPIVAQYLSLPTVFFLHALPCSLEFEATQCPNPFSYVPRPLSSHSDHMTFLQRVKNMLIAFSQNFLCDVVYSPYATLASEFLQREVTVQDLLSSASVWLFRSDFVKDYPRPIMPNMVFVGGINCLHQNPLSQEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# UD14_HUMAN
"MARGLQVPLPRLATGLLLLLSVQPWAESGKVLVVPTDGSPWLSMREALRELHARGHQAVVLTPEVNMHIKEEKFFTLTAYAVPWTQKEFDRVTLGYTQGFFETEHLLKRYSRSMAIMNNVSLALHRCCVELLHNEALIRHLNATSFDVVLTDPVNLCGAVLAKYLSIPAVFFWRYIPCDLDFKGTQCPNPSSYIPKLLTTNSDHMTFLQRVKNMLYPLALSYICHTFSAPYASLASELFQREVSVVDLVSYASVWLFRGDFVMDYPRPIMPNMVFIGGINCANGKPLSQEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# 3BHS2_HUMAN
"MGWSCLVTGAGGLLGQRIVRLLVEEKELKEIRALDKAFRPELREEFSKLQNRTKLTVLEGDILDEPFLKRACQDVSVVIHTACIIDVFGVTHRESIMNVNVKGTQLLLEACVQASVPVFIYTSSIEVAGPNSYKEIIQNGHEEEPLENTWPTPYPYSKKLAEKAVLAANGWNLKNGDTLYTCALRPTYIYGEGGPFLSASINEALNNNGILSSVGKFSTVNPVYVGNVAWAHILALRALRDPKKAPSVRGQFYYISDDTPHQSYDNLNYILSKEFGLRLDSRWSLPLTLMYWIGFLLEVVSFLLSPIYSYQPPFNRHTVTLSNSVFTFSYKKAQRDLAYKPLYSWEEAKQKTVEWVGSLVDRHKETLKSKTQ",
# PPLA_HUMAN
"MEKVQYLTRSAIRRASTIEMPQQARQKLQNLFINFCLILICLLLICIIVMLL",
# FKBP2_HUMAN
"MRLSWFRVLTVLSICLSAVATATGAEGKRKLQIGVKKRVDHCPIKSRKGDVLHMHYTGKLEDGTEFDSSLPQNQPFVFSLGTGQVIKGWDQGLLGMCEGEKRKLVIPSELGYGERGAPPKIPGGATLVFEVELLKIERRTEL",
# CERS1_HUMAN
"MAAAGPAAGPTGPEPMPSYAQLVQRGWGSALAAARGCTDCGWGLARRGLAEHAHLAPPELLLLALGALGWTALRSAATARLFRPLAKRCCLQPRDAAKMPESAWKFLFYLGSWSYSAYLLFGTDYPFFHDPPSVFYDWTPGMAVPRDIAAAYLLQGSFYGHSIYATLYMDTWRKDSVVMLLHHVVTLILIVSSYAFRYHNVGILVLFLHDISDVQLEFTKLNIYFKSRGGSYHRLHALAADLGCLSFGFSWFWFRLYWFPLKVLYATSHCSLRTVPDIPFYFFFNALLLLLTLMNLYWFLYIVAFAAKVLTGQVHELKDLREYDTAEAQSLKPSKAEKPLRNGLVKDKRF",
# CALR_HUMAN
"MLLSVPLLLGLLGLAVAEPAVYFKEQFLDGDGWTSRWIESKHKSDFGKFVLSSGKFYGDEEKDKGLQTSQDARFYALSASFEPFSNKGQTLVVQFTVKHEQNIDCGGGYVKLFPNSLDQTDMHGDSEYNIMFGPDICGPGTKKVHVIFNYKGKNVLINKDIRCKDDEFTHLYTLIVRPDNTYEVKIDNSQVESGSLEDDWDFLPPKKIKDPDASKPEDWDERAKIDDPTDSKPEDWDKPEHIPDPDAKKPEDWDEEMDGEWEPPVIQNPEYKGEWKPRQIDNPDYKGTWIHPEIDNPEYSPDPSIYAYDNFGVLGLDLWQVKSGTIFDNFLITNDEAYAEEFGNETWGVTKAAEKQMKDKQDEEQRLKEEEEDKKRKEEEEAEDKEDDEDKDEDEEDEEDKEEDEEEDVPGQAKDEL",
# CALX_HUMAN
"MEGKWLLCMLLVLGTAIVEAHDGHDDDVIDIEDDLDDVIEEVEDSKPDTTAPPSSPKVTYKAPVPTGEVYFADSFDRGTLSGWILSKAKKDDTDDEIAKYDGKWEVEEMKESKLPGDKGLVLMSRAKHHAISAKLNKPFLFDTKPLIVQYEVNFQNGIECGGAYVKLLSKTPELNLDQFHDKTPYTIMFGPDKCGEDYKLHFIFRHKNPKTGIYEEKHAKRPDADLKTYFTDKKTHLYTLILNPDNSFEILVDQSVVNSGNLLNDMTPPVNPSREIEDPEDRKPEDWDERPKIPDPEAVKPDDWDEDAPAKIPDEEATKPEGWLDDEPEYVPDPDAEKPEDWDEDMDGEWEAPQIANPRCESAPGCGVWQRPVIDNPNYKGKWKPPMIDNPSYQGIWKPRKIPNPDFFEDLEPFRMTPFSAIGLELWSMTSDIFFDNFIICADRRIVDDWANDGWGLKKAADGAAEPGVVGQMIEAAEERPWLWVVYILTVALPVFLVILFCCSGKKQTSGMEYKKTDAPQPDVKEEEEEKEEEKDKGDEEEEGEEKLEEKQKSDAEEDGGTVSQEEEDRKPKAEEDEILNRSPRNRKPRRE",
# PDIA3_HUMAN
"MRLRRLALFPGVALLLAAARLAAASDVLELTDDNFESRISDTGSAGLMLVEFFAPWCGHCKRLAPEYEAAATRLKGIVPLAKVDCTANTNTCNKYGVSGYPTLKIFRDGEEAGAYDGPRTADGIVSHLKKQAGPASVPLRTEEEFKKFISDKDASIVGFFDDSFSEAHSEFLKAASNLRDNYRFAHTNVESLVNEYDDNGEGIILFRPSHLTNKFEDKTVAYTEQKMTSGKIKKFIQENIFGICPHMTEDNKDLIQGKDLLIAYYDVDYEKNAKGSNYWRNRVMMVAKKFLDAGHKLNFAVASRKTFSHELSDFGLESTAGEIPVVAIRTAKGEKFVMQEEFSRDGKALERFLQDYFDGNLKRYLKSEPIPESNDGPVKVVVAENFDEIVNNENKDVLIEFYAPWCGHCKNLEPKYKELGEKLSKDPNIVIAKMDATANDVPSPYEVRGFPTIYFSPANKKLNPKKYEGGRELSDFISYLQREATNPPVIQEEKPKKKKKAQEDL",
# HLAF_HUMAN
"MAPRSLLLLLSGALALTDTWAGSHSLRYFSTAVSRPGRGEPRYIAVEYVDDTQFLRFDSDAAIPRMEPREPWVEQEGPQYWEWTTGYAKANAQTDRVALRNLLRRYNQSEAGSHTLQGMNGCDMGPDGRLLRGYHQHAYDGKDYISLNEDLRSWTAADTVAQITQRFYEAEEYAEEFRTYLEGECLELLRRYLENGKETLQRADPPKAHVAHHPISDHEATLRCWALGFYPAEITLTWQRDGEEQTQDTELVETRPAGDGTFQKWAAVVVPPGEEQRYTCHVQHEGLPQPLILRWEQSPQPTIPIVGIVAGLVVLGAVVTGAVVAAVMWRKKSSDRNRGSYSQAAV",
# CASQ1_HUMAN
"MSATDRMGPRAVPGLRLALLLLLVLGTPKSGVQGQEGLDFPEYDGVDRVINVNAKNYKNVFKKYEVLALLYHEPPEDDKASQRQFEMEELILELAAQVLEDKGVGFGLVDSEKDAAVAKKLGLTEVDSMYVFKGDEVIEYDGEFSADTIVEFLLDVLEDPVELIEGERELQAFENIEDEIKLIGYFKSKDSEHYKAFEDAAEEFHPYIPFFATFDSKVAKKLTLKLNEIDFYEAFMEEPVTIPDKPNSEEEIVNFVEEHRRSTLRKLKPESMYETWEDDMDGIHIVAFAEEADPDGFEFLETLKAVAQDNTENPDLSIIWIDPDDFPLLVPYWEKTFDIDLSAPQIGVVNVTDADSVWMEMDDEEDLPSAEELEDWLEDVLEGEINTEDDDDDDDD",
# ERD22_HUMAN
"MNIFRLTGDLSHLAAIVILLLKIWKTRSCAGISGKSQLLFALVFTTRYLDLFTSFISLYNTSMKVIYLACSYATVYLIYLKFKATYDGNHDTFRVEFLVVPVGGLSFLVNHDFSPLEILWTFSIYLESVAILPQLFMISKTGEAETITTHYLFFLGLYRALYLVNWIWRFYFEGFFDLIAVVAGVVQTILYCDFFYLYITKVLKGKKLSLPA",
# PGH2_HUMAN
"MLARALLLCAVLALSHTANPCCSHPCQNRGVCMSVGFDQYKCDCTRTGFYGENCSTPEFLTRIKLFLKPTPNTVHYILTHFKGFWNVVNNIPFLRNAIMSYVLTSRSHLIDSPPTYNADYGYKSWEAFSNLSYYTRALPPVPDDCPTPLGVKGKKQLPDSNEIVEKLLLRRKFIPDPQGSNMMFAFFAQHFTHQFFKTDHKRGPAFTNGLGHGVDLNHIYGETLARQRKLRLFKDGKMKYQIIDGEMYPPTVKDTQAEMIYPPQVPEHLRFAVGQEVFGLVPGLMMYATIWLREHNRVCDVLKQEHPEWGDEQLFQTSRLILIGETIKIVIEDYVQHLSGYHFKLKFDPELLFNKQFQYQNRIAAEFNTLYHWHPLLPDTFQIHDQKYNYQQFIYNNSILLEHGITQFVESFTRQIAGRVAGGRNVPPAVQKVSQASIDQSRQMKYQSFNEYRKRFMLKPYESFEELTGEKEMSAELEALYGDIDAVELYPALLVEKPRPDAIFGETMVEVGAPFSLKGLMGNVICSPAYWKPSTFGGEVGFQIINTASIQSLICNNVKGCPFTSFSVPDPELIKTVTINASSSRSGLDDINPTVLLKERSTEL",
# UD13_HUMAN
"MATGLQVPLPWLATGLLLLLSVQPWAESGKVLVVPIDGSHWLSMREVLRELHARGHQAVVLTPEVNMHIKEENFFTLTTYAISWTQDEFDRHVLGHTQLYFETEHFLKKFFRSMAMLNNMSLVYHRSCVELLHNEALIRHLNATSFDVVLTDPVNLCAAVLAKYLSIPTVFFLRNIPCDLDFKGTQCPNPSSYIPRLLTTNSDHMTFMQRVKNMLYPLALSYICHAFSAPYASLASELFQREVSVVDILSHASVWLFRGDFVMDYPRPIMPNMVFIGGINCANRKPLSQEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# SOAT1_HUMAN
"MVGEEKMSLRNRLSKSRENPEEDEDQRNPAKESLETPSNGRIDIKQLIAKKIKLTAEAEELKPFFMKEVGSHFDDFVTNLIEKSASLDNGGCALTTFSVLEGEKNNHRAKDLRAPPEQGKIFIARRSLLDELLEVDHIRTIYHMFIALLILFILSTLVVDYIDEGRLVLEFSLLSYAFGKFPTVVWTWWIMFLSTFSVPYFLFQHWATGYSKSSHPLIRSLFHGFLFMIFQIGVLGFGPTYVVLAYTLPPASRFIIIFEQIRFVMKAHSFVRENVPRVLNSAKEKSSTVPIPTVNQYLYFLFAPTLIYRDSYPRNPTVRWGYVAMKFAQVFGCFFYVYYIFERLCAPLFRNIKQEPFSARVLVLCVFNSILPGVLILFLTFFAFLHCWLNAFAEMLRFGDRMFYKDWWNSTSYSNYYRTWNVVVHDWLYYYAYKDFLWFFSKRFKSAAMLAVFAVSAVVHEYALAVCLSFFYPVLFVLFMFFGMAFNFIVNDSRKKPIWNVLMWTSLFLGNGVLLCFYSQEWYARQHCPLKNPTFLDYVRPRSWTCRYVF",
# DHB3_HUMAN
"MGDVLEQFFILTGLLVCLACLAKCVRFSRCVLLNYWKVLPKSFLRSMGQWAVITGAGDGIGKAYSFELAKRGLNVVLISRTLEKLEAIATEIERTTGRSVKIIQADFTKDDIYEHIKEKLAGLEIGILVNNVGMLPNLLPSHFLNAPDEIQSLIHCNITSVVKMTQLILKHMESRQKGLILNISSGIALFPWPLYSMYSASKAFVCAFSKALQEEYKAKEVIIQVLTPYAVSTAMTKYLNTNVITKTADEFVKESLNYVTIGGETCGCLAHEILAGFLSLIPAWAFYSGAFQRLLLTHYVAYLKLNTKVR",
# FDFT_HUMAN
"MEFVKCLGHPEEFYNLVRFRIGGKRKVMPKMDQDSLSSSLKTCYKYLNQTSRSFAAVIQALDGEMRNAVCIFYLVLRALDTLEDDMTISVEKKVPLLHNFHSFLYQPDWRFMESKEKDRQVLEDFPTISLEFRNLAEKYQTVIADICRRMGIGMAEFLDKHVTSEQEWDKYCHYVAGLVGIGLSRLFSASEFEDPLVGEDTERANSMGLFLQKTNIIRDYLEDQQGGREFWPQEVWSRYVKKLGDFAKPENIDLAVQCLNELITNALHHIPDVITYLSRLRNQSVFNFCAIPQVMAIATLAACYNNQQVFKGAVKIRKGQAVTLMMDATNMPAVKAIIYQYMEEIYHRIPDSDPSSSKTRQIISTIRTQNLPNCQLISRSHYSPIYLSFVMLLAALSWQYLTTLSQVTEDYVQTGEH",
# INGR2_HUMAN
"MRPTLLWSLLLLLGVFAAAAAAPPDPLSQLPAPQHPKIRLYNAEQVLSWEPVALSNSTRPVVYQVQFKYTDSKWFTADIMSIGVNCTQITATECDFTAASPSAGFPMDFNVTLRLRAELGALHSAWVTMPWFQHYRNVTVGPPENIEVTPGEGSLIIRFSSPFDIADTSTAFFCYYVHYWEKGGIQQVKGPFRSNSISLDNLKPSRVYCLQVQAQLLWNKSNIFRVGHLSNISCYETMADASTELQQVILISVGTFSLLSVLAGACFFLVLKYRGLIKYWFHTPPSIPLQIEEYLKDPTQPILEALDKDSSPKDDVWDSVSIISFPEKEQEDVLQTL",
# OST48_HUMAN
"MGYFRCARAGSFGRRRKMEPSTAARAWALFWLLLPLLGAVCASGPRTLVLLDNLNVRETHSLFFRSLKDRGFELTFKTADDPSLSLIKYGEFLYDNLIIFSPSVEDFGGNINVETISAFIDGGGSVLVAASSDIGDPLRELGSECGIEFDEEKTAVIDHHNYDISDLGQHTLIVADTENLLKAPTIVGKSSLNPILFRGVGMVADPDNPLVLDILTGSSTSYSFFPDKPITQYPHAVGKNTLLIAGLQARNNARVIFSGSLDFFSDSFFNSAVQKAAPGSQRYSQTGNYELAVALSRWVFKEEGVLRVGPVSHHRVGETAPPNAYTVTDLVEYSIVIQQLSNGKWVPFDGDDIQLEFVRIDPFVRTFLKKKGGKYSVQFKLPDVYGVFQFKVDYNRLGYTHLYSSTQVSVRPLQHTQYERFIPSAYPYYASAFSMMLGLFIFSIVFLHMKEKEKSD",
# SSRA_HUMAN
"MRLLPRLLLLLLLVFPATVLFRGGPRGLLAVAQDLTEDEETVEDSIIEDEDDEAEVEEDEPTDLVEDKEEEDVSGEPEASPSADTTILFVKGEDFPANNIVKFLVGFTNKGTEDFIVESLDASFRYPQDYQFYIQNFTALPLNTVVPPQRQATFEYSFIPAEPMGGRPFGLVINLNYKDLNGNVFQDAVFNQTVTVIEREDGLDGETIFMYMFLAGLGLLVIVGLHQLLESRKRKRPIQKVEMGTSSQNDVDMSWIPQETLNQINKASPRRLPRKRAQKRSVGSDE",
# SSRB_HUMAN
"MRLLSFVVLALFAVTQAEEGARLLASKSLLNRYAVEGRDLTLQYNIYNVGSSAALDVELSDDSFPPEDFGIVSGMLNVKWDRIAPASNVSHTVVLRPLKAGYFNFTSATITYLAQEDGPVVIGSTSAPGQGGILAQREFDRRFSPHFLDWAAFGVMTLPSIGIPLLLWYSSKRKYDTPKTKKN",
# CAMLG_HUMAN
"MESMAVATDGGERPGVPAGSGLSASQRRAELRRRKLLMNSEQRINRIMGFHRPGSGAEEESQTKSKQQDSDKLNSLSVPSVSKRVVLGDSVSTGTTDQQGGVAEVKGTQLGDKLDSFIKPPECSSDVNLELRQRNRGDLTADSVQRGSRHGLEQYLSRFEEAMKLRKQLISEKPSQEDGNTTEEFDSFRIFRLVGCALLALGVRAFVCKYLSIFAPFLTLQLAYMGLYKYFPKSEKKIKTTVLTAALLLSGIPAEVINRSMDTYSKMGEVFTDLCVYFFTFIFCHELLDYWGSEVP",
# FMO5_HUMAN
"MTKKRIAVIGGGVSGLSSIKCCVEEGLEPVCFERTDDIGGLWRFQENPEEGRASIYKSVIINTSKEMMCFSDYPIPDHYPNFMHNAQVLEYFRMYAKEFDLLKYIRFKTTVCSVKKQPDFATSGQWEVVTESEGKKEMNVFDGVMVCTGHHTNAHLPLESFPGIEKFKGQYFHSRDYKNPEGFTGKRVIIIGIGNSGGDLAVEISQTAKQVFLSTRRGAWILNRVGDYGYPADVLFSSRLTHFIWKICGQSLANKYLEKKINQRFDHEMFGLKPKHRALSQHPTLNDDLPNRIISGLVKVKGNVKEFTETAAIFEDGSREDDIDAVIFATGYSFDFPFLEDSVKVVKNKISLYKKVFPPNLERPTLAIIGLIQPLGAIMPISELQGRWATQVFKGLKTLPSQSEMMAEISKAQEEIDKRYVESQRHTIQGDYIDTMEELADLVGVRPNLLSLAFTDPKLALHLLLGPCTPIHYRVQGPGKWDGARKAILTTDDRIRKPLMTRVVERSSSMTSTMTIGKFMLALAFFAIIIAYF",
# CASP4_HUMAN
"MAEGNHRKKPLKVLESLGKDFLTGVLDNLVEQNVLNWKEEEKKKYYDAKTEDKVRVMADSMQEKQRMAGQMLLQTFFNIDQISPNKKAHPNMEAGPPESGESTDALKLCPHEEFLRLCKERAEEIYPIKERNNRTRLALIICNTEFDHLPPRNGADFDITGMKELLEGLDYSVDVEENLTARDMESALRAFATRPEHKSSDSTFLVLMSHGILEGICGTVHDEKKPDVLLYDTIFQIFNNRNCLSLKDKPKVIIVQACRGANRGELWVRDSPASLEVASSQSSENLEEDAVYKTHVEKDFIAFCSSTPHNVSWRDSTMGSIFITQLITCFQKYSWCCHLEEVFRKVQQSFETPRAKAQMPTIERLSMTRYFYLFPGN",
# TMEDA_HUMAN
"MSGLSGPPARRGPFPLALLLLFLLGPRLVLAISFHLPINSRKCLREEIHKDLLVTGAYEISDQSGGAGGLRSHLKITDSAGHILYSKEDATKGKFAFTTEDYDMFEVCFESKGTGRIPDQLVILDMKHGVEAKNYEEIAKVEKLKPLEVELRRLEDLSESIVNDFAYMKKREEEMRDTNESTNTRVLYFSIFSMFCLIGLATWQVFYLRRFFKAKKLIE",
# PSN1_HUMAN
"MTELPAPLSYFQNAQMSEDNHLSNTVRSQNDNRERQEHNDRRSLGHPEPLSNGRPQGNSRQVVEQDEEEDEELTLKYGAKHVIMLFVPVTLCMVVVVATIKSVSFYTRKDGQLIYTPFTEDTETVGQRALHSILNAAIMISVIVVMTILLVVLYKYRCYKVIHAWLIISSLLLLFFFSFIYLGEVFKTYNVAVDYITVALLIWNFGVVGMISIHWKGPLRLQQAYLIMISALMALVFIKYLPEWTAWLILAVISVYDLVAVLCPKGPLRMLVETAQERNETLFPALIYSSTMVWLVNMAEGDPEAQRRVSKNSKYNAESTERESQDTVAENDDGGFSEEWEAQRDSHLGPHRSTPESRAAVQELSSSILAGEDPEERGVKLGLGDFIFYSVLVGKASATASGDWNTTIACFVAILIGLCLTLLLLAIFKKALPALPISITFGLVFYFATDYLVQPFMDQLAFHQFYI",
# PSN2_HUMAN
"MLTFMASDSEEEVCDERTSLMSAESPTPRSCQEGRQGPEDGENTAQWRSQENEEDGEEDPDRYVCSGVPGRPPGLEEELTLKYGAKHVIMLFVPVTLCMIVVVATIKSVRFYTEKNGQLIYTPFTEDTPSVGQRLLNSVLNTLIMISVIVVMTIFLVVLYKYRCYKFIHGWLIMSSLMLLFLFTYIYLGEVLKTYNVAMDYPTLLLTVWNFGAVGMVCIHWKGPLVLQQAYLIMISALMALVFIKYLPEWSAWVILGAISVYDLVAVLCPKGPLRMLVETAQERNEPIFPALIYSSAMVWTVGMAKLDPSSQGALQLPYDPEMEEDSYDSFGEPSYPEVFEPPLTGYPGEELEEEEERGVKLGLGDFIFYSVLVGKAAATGSGDWNTTLACFVAILIGLCLTLLLLAVFKKALPALPISITFGLIFYFSTDNLVRPFMDTLASHQLYI",
# LRBA_HUMAN
"MASEDNRVPSPPPTGDDGGGGGREETPTEGGALSLKPGLPIRGIRMKFAVLTGLVEVGEVSNRDIVETVFNLLVGGQFDLEMNFIIQEGESINCMVDLLEKCDITCQAEVWSMFTAILKKSIRNLQVCTEVGLVEKVLGKIEKVDNMIADLLVDMLGVLASYNLTVRELKLFFSKLQGDKGRWPPHAGKLLSVLKHMPQKYGPDAFFNFPGKSAAAIALPPIAKWPYQNGFTFHTWLRMDPVNNINVDKDKPYLYCFRTSKGLGYSAHFVGGCLIVTSIKSKGKGFQHCVKFDFKPQKWYMVTIVHIYNRWKNSELRCYVNGELASYGEITWFVNTSDTFDKCFLGSSETADANRVFCGQMTAVYLFSEALNAAQIFAIYQLGLGYKGTFKFKAESDLFLAEHHKLLLYDGKLSSAIAFTYNPRATDAQLCLESSPKDNPSIFVHSPHALMLQDVKAVLTHSIQSAMHSIGGVQVLFPLFAQLDYRQYLSDEIDLTICSTLLAFIMELLKNSIAMQEQMLACKGFLVIGYSLEKSSKSHVSRAVLELCLAFSKYLSNLQNGMPLLKQLCDHVLLNPAIWIHTPAKVQLMLYTYLSTEFIGTVNIYNTIRRVGTVLLIMHTLKYYYWAVNPQDRSGITPKGLDGPRPNQKEMLSLRAFLLMFIKQLVMKDSGVKEDELQAILNYLLTMHEDDNLMDVLQLLVALMSEHPNSMIPAFDQRNGLRVIYKLLASKSEGIRVQALKAMGYFLKHLAPKRKAEVMLGHGLFSLLAERLMLQTNLITMTTYNVLFEILIEQIGTQVIHKQHPDPDSSVKIQNPQILKVIATLLRNSPQCPESMEVRRAFLSDMIKLFNNSRENRRSLLQCSVWQEWMLSLCYFNPKNSDEQKITEMVYAIFRILLYHAVKYEWGGWRVWVDTLSITHSKVTFEIHKENLANIFREQQGKVDEEIGLCSSTSVQAASGIRRDINVSVGSQQPDTKDSPVCPHFTTNGNENSSIEKTSSLESASNIELQTTNTSYEEMKAEQENQELPDEGTLEETLTNETRNADDLEVSSDIIEAVAISSNSFITTGKDSMTVSEVTASISSPSEEDASEMPEFLDKSIVEEEEDDDYVELKVEGSPTEEANLPTELQDNSLSPAASEAGEKLDMFGNDDKLIFQEGKPVTEKQTDTETQDSKDSGIQTMTASGSSAMSPETTVSQIAVESDLGQMLEEGKKATNLTRETKLINDCHGSVSEASSEQKIAKLDVSNVATDTERLELKASPNVEAPQPHRHVLEISRQHEQPGQGIAPDAVNGQRRDSRSTVFRIPEFNWSQMHQRLLTDLLFSIETDIQMWRSHSTKTVMDFVNSSDNVIFVHNTIHLISQVMDNMVMACGGILPLLSAATSATHELENIEPTQGLSIEASVTFLQRLISLVDVLIFASSLGFTEIEAEKSMSSGGILRQCLRLVCAVAVRNCLECQQHSQLKTRGDKALKPMHSLIPLGKSAAKSPVDIVTGGISPVRDLDRLLQDMDINRLRAVVFRDIEDSKQAQFLALAVVYFISVLMVSKYRDILEPQNERHSQSCTETGSENENVSLSEITPAAFSTLTTASVEESESTSSARRRDSGIGEETATGLGSHVEVTPHTAPPGVSAGPDAISEVLSTLSLEVNKSPETKNDRGNDLDTKATPSVSVSKNVNVKDILRSLVNIPADGVTVDPALLPPACLGALGDLSVEQPVQFRSFDRSVIVAAKKSAVSPSTFNTSIPTNAVSVVSSVDSAQASDMGGESPGSRSSNAKLPSVPTVDSVSQDPVSNMSITERLEHALEKAAPLLREIFVDFAPFLSRTLLGSHGQELLIEGTSLVCMKSSSSVVELVMLLCSQEWQNSIQKNAGLAFIELVNEGRLLSQTMKDHLVRVANEAEFILSRQRAEDIHRHAEFESLCAQYSADKREDEKMCDHLIRAAKYRDHVTATQLIQKIINILTDKHGAWGNSAVSRPLEFWRLDYWEDDLRRRRRFVRNPLGSTHPEATLKTAVEHVCIFKLRENSKATDEDILAKGKQSIRSQALGNQNSENEILLEGDDDTLSSVDEKDLENLAGPVSLSTPAQLVAPSVVVKGTLSVTSSELYFEVDEEDPNFKKIDPKILAYTEGLHGKWLFTEIRSIFSRRYLLQNTALEIFMANRVAVMFNFPDPATVKKVVNYLPRVGVGTSFGLPQTRRISLASPRQLFKASNMTQRWQHREISNFEYLMFLNTIAGRSYNDLNQYPVFPWVITNYESEELDLTLPTNFRDLSKPIGALNPKRAAFFAERYESWEDDQVPKFHYGTHYSTASFVLAWLLRIEPFTTYFLNLQGGKFDHADRTFSSISRAWRNSQRDTSDIKELIPEFYYLPEMFVNFNNYNLGVMDDGTVVSDVELPPWAKTSEEFVHINRLALESEFVSCQLHQWIDLIFGYKQQGPEAVRALNVFYYLTYEGAVNLNSITDPVLREAVEAQIRSFGQTPSQLLIEPHPPRGSAMQVSPLMFTDKAQQDVIMVLKFPSNSPVTHVAANTQPGLATPAVITVTANRLFAVNKWHNLPAHQGAVQDQPYQLPVEIDPLIASNTGMHRRQITDLLDQSIQVHSQCFVITSDNRYILVCGFWDKSFRVYSTDTGRLIQVVFGHWDVVTCLARSESYIGGNCYILSGSRDATLLLWYWNGKCSGIGDNPGSETAAPRAILTGHDYEVTCAAVCAELGLVLSGSQEGPCLIHSMNGDLLRTLEGPENCLKPKLIQASREGHCVIFYENGLFCTFSVNGKLQATMETDDNIRAIQLSRDGQYLLTGGDRGVVVVRQVSDLKQLFAYPGCDAGIRAMALSYDQRCIISGMASGSIVLFYNDFNRWHHEYQTRY",
# BAP31_HUMAN
"MSLQWTAVATFLYAEVFVVLLLCIPFISPKRWQKIFKSRLVELLVSYGNTFFVVLIVILVLLVIDAVREIRKYDDVTEKVNLQNNPGAMEHFHMKLFRAQRNLYIAGFSLLLSFLLRRLVTLISQQATLLASNEAFKKQAESASEAAKKYMEENDQLKKGAAVDGGKLDVGNAEVKLEEENRSLKADLQKLKDELASTKQKLEKAENQVLAMRKQSEGLTKEYDRLLEEHAKLQAAVDGPMDKKEE",
# PLAT3_HUMAN
"MRAPIPEPKPGDLIEIFRPFYRHWAIYVGDGYVVHLAPPSEVAGAGAASVMSALTDKAIVKKELLYDVAGSDKYQVNNKHDDKYSPLPCSKIIQRAEELVGQEVLYKLTSENCEHFVNELRYGVARSDQVRDVIIAASVAGMGLAAMSLIGVMFSRNKRQKQ",
# BI1_HUMAN
"MNIFDRKINFDALLKFSHITPSTQQHLKKVYASFALCMFVAAAGAYVHMVTHFIQAGLLSALGSLILMIWLMATPHSHETEQKRLGLLAGFAFLTGVGLGPALEFCIAVNPSILPTAFMGTAMIFTCFTLSALYARRRSYLFLGGILMSALSLLLLSSLGNVFFGSIWLFQANLYVGLVVMCGFVLFDTQLIIEKAEHGDQDYIWHCIDLFLDFITVFRKLMMILAMNEKDKKKEKK",
# TERA_HUMAN
"MASGADSKGDDLSTAILKQKNRPNRLIVDEAINEDNSVVSLSQPKMDELQLFRGDTVLLKGKKRREAVCIVLSDDTCSDEKIRMNRVVRNNLRVRLGDVISIQPCPDVKYGKRIHVLPIDDTVEGITGNLFEVYLKPYFLEAYRPIRKGDIFLVRGGMRAVEFKVVETDPSPYCIVAPDTVIHCEGEPIKREDEEESLNEVGYDDIGGCRKQLAQIKEMVELPLRHPALFKAIGVKPPRGILLYGPPGTGKTLIARAVANETGAFFFLINGPEIMSKLAGESESNLRKAFEEAEKNAPAIIFIDELDAIAPKREKTHGEVERRIVSQLLTLMDGLKQRAHVIVMAATNRPNSIDPALRRFGRFDREVDIGIPDATGRLEILQIHTKNMKLADDVDLEQVANETHGHVGADLAALCSEAALQAIRKKMDLIDLEDETIDAEVMNSLAVTMDDFRWALSQSNPSALRETVVEVPQVTWEDIGGLEDVKRELQELVQYPVEHPDKFLKFGMTPSKGVLFYGPPGCGKTLLAKAIANECQANFISIKGPELLTMWFGESEANVREIFDKARQAAPCVLFFDELDSIAKARGGNIGDGGGAADRVINQILTEMDGMSTKKNVFIIGATNRPDIIDPAILRPGRLDQLIYIPLPDEKSRVAILKANLRKSPVAKDVDLEFLAKMTNGFSGADLTEICQRACKLAIRESIESEIRRERERQTNPSAMEVEEDDPVPEIRRDHFEEAMRFARRSVSDNDIRKYEMFAQTLQQSRGFGSFRFPSGNQGGAGPSQGSGGGTGGSVYTEDNDDDLYG",
# TM50B_HUMAN
"MAGFLDNFRWPECECIDWSERRNAVASVVAGILFFTGWWIMIDAAVVYPKPEQLNHAFHTCGVFSTLAFFMINAVSNAQVRGDSYESGCLGRTGARVWLFIGFMLMFGSLIASMWILFGAYVTQNTDVYPGLAVFFQNALIFFSTLIYKFGRTEELWT",
# TMM33_HUMAN
"MADTTPNGPQGAGAVQFMMTNKLDTAMWLSRLFTVYCSALFVLPLLGLHEAASFYQRALLANALTSALRLHQRLPHFQLSRAFLAQALLEDSCHYLLYSLIFVNSYPVTMSIFPVLLFSLLHAATYTKKVLDARGSNSLPLLRSVLDKLSANQQNILKFIACNEIFLMPATVFMLFSGQGSLLQPFIYYRFLTLRYSSRRNPYCRTLFNELRIVVEHIIMKPACPLFVRRLCLQSIAFISRLAPTVP",
# TMPS3_HUMAN
"MGENDPPAVEAPFSFRSLFGLDDLKISPVAPDADAVAAQILSLLPLKFFPIIVIGIIALILALAIGLGIHFDCSGKYRCRSSFKCIELIARCDGVSDCKDGEDEYRCVRVGGQNAVLQVFTAASWKTMCSDDWKGHYANVACAQLGFPSYVSSDNLRVSSLEGQFREEFVSIDHLLPDDKVTALHHSVYVREGCASGHVVTLQCTACGHRRGYSSRIVGGNMSLLSQWPWQASLQFQGYHLCGGSVITPLWIITAAHCVYDLYLPKSWTIQVGLVSLLDNPAPSHLVEKIVYHSKYKPKRLGNDIALMKLAGPLTFNEMIQPVCLPNSEENFPDGKVCWTSGWGATEDGAGDASPVLNHAAVPLISNKICNHRDVYGGIISPSMLCAGYLTGGVDSCQGDSGGPLVCQERRLWKLVGATSFGIGCAEVNKPGVYTRVTSFLDWIHEQMERDLKT",
# SC61B_HUMAN
"MPGPTPSGTNVGSSGRSPSKAVAARAAGSTVRQRKNASCGTRSAGRTTSAGTGGMWRFYTEDSPGLKVGPVPVLVMSLLFIASVFMLHIWGKYTRS",
# SPCS3_HUMAN
"MNTVLSRANSLFAFSLSVMAALTFGCFITTAFKDRSVPVRLHVSRIMLKNVEDFTGPRERSDLGFITFDITADLENIFDWNVKQLFLYLSAEYSTKNNALNQVVLWDKIVLRGDNPKLLLKDMKTKYFFFDDGNGLKGNRNVTLTLSWNVVPNAGILPLVTGSGHVSVPFPDTYEITKSY",
# TM258_HUMAN
"MELEAMSRYTSPVNPAVFPHLTVVLLAIGMFFTAWFFVYEVTSTKYTRDIYKELLISLVASLFMGFGVLFLLLWVGIYV",
# SELT_HUMAN
"MRLLLLLLVAASAMVRSEASANLGGVPSKRLKMQYATGPLLKFQICVSUGYRRVFEEYMRVISQRYPDIRIEGENYLPQPIYRHIASFLSVFKLVLIGLIIVGKDPFAFFGMQAPSIWQWGQENKVYACMMVFFLSNMIENQCMSTGAFEITLNDVPVWSKLESGHLPSMQQLVQILDNEMKLNVHMDSIPHHRS",
# RL40_HUMAN
"MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGIIEPSLRQLAQKYNCDKMICRKCYARLHPRAVNCRKKKCGHTNNLRPKKKVK",
# S35A2_HUMAN
"MAAVGAGGSTAAPGPGAVSAGALEPGTASAAHRRLKYISLAVLVVQNASLILSIRYARTLPGDRFFATTAVVMAEVLKGLTCLLLLFAQKRGNVKHLVLFLHEAVLVQYVDTLKLAVPSLIYTLQNNLQYVAISNLPAATFQVTYQLKILTTALFSVLMLNRSLSRLQWASLLLLFTGVAIVQAQQAGGGGPRPLDQNPGAGLAAVVASCLSSGFAGVYFEKILKGSSGSVWLRNLQLGLFGTALGLVGLWWAEGTAVATRGFFFGYTPAVWGVVLNQAFGGLLVAVVVKYADNILKGFATSLSIVLSTVASIRLFGFHVDPLFALGAGLVIGAVYLYSLPRGAAKAIASASASASGPCVHQQPPGQPPPPQLSSHRGDLITEPFLPKLLTKVKGS",
# AT11A_HUMAN
"MDCSLVRTLVHRYCAGEENWVDSRTIYVGHREPPPGAEAYIPQRYPDNRIVSSKYTFWNFIPKNLFEQFRRVANFYFLIIFLVQLIIDTPTSPVTSGLPLFFVITVTAIKQGYEDWLRHKADNAMNQCPVHFIQHGKLVRKQSRKLRVGDIVMVKEDETFPCDLIFLSSNRGDGTCHVTTASLDGESSHKTHYAVQDTKGFHTEEDIGGLHATIECEQPQPDLYKFVGRINVYSDLNDPVVRPLGSENLLLRGATLKNTEKIFGVAIYTGMETKMALNYQSKSQKRSAVEKSMNAFLIVYLCILISKALINTVLKYMWQSEPFRDEPWYNQKTESERQRNLFLKAFTDFLAFMVLFNYIIPVSMYVTVEMQKFLGSYFITWDEDMFDEETGEGPLVNTSDLNEELGQVEYIFTDKTGTLTENNMEFKECCIEGHVYVPHVICNGQVLPESSGIDMIDSSPSVNGREREELFFRALCLCHTVQVKDDDSVDGPRKSPDGGKSCVYISSSPDEVALVEGVQRLGFTYLRLKDNYMEILNRENHIERFELLEILSFDSVRRRMSVIVKSATGEIYLFCKGADSSIFPRVIEGKVDQIRARVERNAVEGLRTLCVAYKRLIQEEYEGICKLLQAAKVALQDREKKLAEAYEQIEKDLTLLGATAVEDRLQEKAADTIEALQKAGIKVWVLTGDKMETAAATCYACKLFRRNTQLLELTTKRIEEQSLHDVLFELSKTVLRHSGSLTRDNLSGLSADMQDYGLIIDGAALSLIMKPREDGSSGNYRELFLEICRSCSAVLCCRMAPLQKAQIVKLIKFSKEHPITLAIGDGANDVSMILEAHVGIGVIGKEGRQAARNSDYAIPKFKHLKKMLLVHGHFYYIRISELVQYFFYKNVCFIFPQFLYQFFCGFSQQTLYDTAYLTLYNISFTSLPILLYSLMEQHVGIDVLKRDPTLYRDVAKNALLRWRVFIYWTLLGLFDALVFFFGAYFVFENTTVTSNGQIFGNWTFGTLVFTVMVFTVTLKLALDTHYWTWINHFVIWGSLLFYVVFSLLWGGVIWPFLNYQRMYYVFIQMLSSGPAWLAIVLLVTISLLPDVLKKVLCRQLWPTATERVQTKSQCLSVEQSTIFMLSQTSSSLSF",
# CAV1_HUMAN
"MSGGKYVDSEGHLYTVPIREQGNIYKPNNKAMADELSEKQVYDAHTKEIDLVNRDPKHLNDDVVKIDFEDVIAEPEGTHSFDGIWKASFTTFTVTKYWFYRLLSALFGIPMALIWGIYFAILSFLHIWAVVPCIKSFLIEIQCISRVYSIYVHTVCDPLFEAVGKIFSNVRINLQKEI",
# TAP1_HUMAN
"MASSRCPAPRGCRCLPGASLAWLGTVLLLLADWVLLRTALPRIFSLLVPTALPLLRVWAVGLSRWAVLWLGACGVLRATVGSKSENAGAQGWLAALKPLAAALGLALPGLALFRELISWGAPGSADSTRLLHWGSHPTAFVVSYAAALPAAALWHKLGSLWVPGGQGGSGNPVRRLLGCLGSETRRLSLFLVLVVLSSLGEMAIPFFTGRLTDWILQDGSADTFTRNLTLMSILTIASAVLEFVGDGIYNNTMGHVHSHLQGEVFGAVLRQETEFFQQNQTGNIMSRVTEDTSTLSDSLSENLSLFLWYLVRGLCLLGIMLWGSVSLTMVTLITLPLLFLLPKKVGKWYQLLEVQVRESLAKSSQVAIEALSAMPTVRSFANEEGEAQKFREKLQEIKTLNQKEAVAYAVNSWTTSISGMLLKVGILYIGGQLVTSGAVSSGNLVTFVLYQMQFTQAVEVLLSIYPRVQKAVGSSEKIFEYLDRTPRCPPSGLLTPLHLEGLVQFQDVSFAYPNRPDVLVLQGLTFTLRPGEVTALVGPNGSGKSTVAALLQNLYQPTGGQLLLDGKPLPQYEHRYLHRQVAAVGQEPQVFGRSLQENIAYGLTQKPTMEEITAAAVKSGAHSFISGLPQGYDTEVDEAGSQLSGGQRQAVALARALIRKPCVLILDDATSALDANSQLQVEQLLYESPERYSRSVLLITQHLSLVEQADHILFLEGGAIREGGTHQQLMEKKGCYWAMVQAPADAPE",
# TAP2_HUMAN
"MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRLGGLWGLLKLRGLLGFVGTLLLPLCLATPLTVSLRALVAGASRAPPARVASAPWSWLLVGYGAAGLSWSLWAVLSPPGAQEKEQDQVNNKVLMWRLLKLSRPDLPLLVAAFFFLVLAVLGETLIPHYSGRVIDILGGDFDPHAFASAIFFMCLFSFGSSLSAGCRGGCFTYTMSRINLRIREQLFSSLLRQDLGFFQETKTGELNSRLSSDTTLMSNWLPLNANVLLRSLVKVVGLYGFMLSISPRLTLLSLLHMPFTIAAEKVYNTRHQEVLREIQDAVARAGQVVREAVGGLQTVRSFGAEEHEVCRYKEALEQCRQLYWRRDLERALYLLVRRVLHLGVQMLMLSCGLQQMQDGELTQGSLLSFMIYQESVGSYVQTLVYIYGDMLSNVGAAEKVFSYMDRQPNLPSPGTLAPTTLQGVVKFQDVSFAYPNRPDRPVLKGLTFTLRPGEVTALVGPNGSGKSTVAALLQNLYQPTGGQVLLDEKPISQYEHCYLHSQVVSVGQEPVLFSGSVRNNIAYGLQSCEDDKVMAAAQAAHADDFIQEMEHGIYTDVGEKGSQLAAGQKQRLAIARALVRDPRVLILDEATSALDVQCEQALQDWNSRGDRTVLVIAHRLQTVQRAHQILVLQEGKLQKLAQL",
# UBXN1_HUMAN
"MAELTALESLIEMGFPRGRAEKALALTGNQGIEAAMDWLMEHEDDPDVDEPLETPLGHILGREPTSSEQGGLEGSGSAAGEGKPALSEEERQEQTKRMLELVAQKQREREEREEREALERERQRRRQGQELSAARQRLQEDEMRRAAEERRREKAEELAARQRVREKIERDKAERAKKYGGSVGSQPPPVAPEPGPVPSSPSQEPPTKREYDQCRIQVRLPDGTSLTQTFRAREQLAAVRLYVELHRGEELGGGQDPVQLLSGFPRRAFSEADMERPLQELGLVPSAVLIVAKKCPS",
# PLP2_HUMAN
"MADSERLSAPGCWAACTNFSRTRKGILLFAEIILCLVILICFSASTPGYSSLSVIEMILAAIFFVVYMCDLHTKIPFINWPWSDFFRTLIAAILYLITSIVVLVERGNHSKIVAGVLGLIATCLFGYDAYVTFPVRQPRHTAAPTDPADGPV",
# NMDZ1_HUMAN
"MSTMRLLTLALLFSCSVARAACDPKIVNIGAVLSTRKHEQMFREAVNQANKRHGSWKIQLNATSVTHKPNAIQMALSVCEDLISSQVYAILVSHPPTPNDHFTPTPVSYTAGFYRIPVLGLTTRMSIYSDKSIHLSFLRTVPPYSHQSSVWFEMMRVYSWNHIILLVSDDHEGRAAQKRLETLLEERESKAEKVLQFDPGTKNVTALLMEAKELEARVIILSASEDDAATVYRAAAMLNMTGSGYVWLVGEREISGNALRYAPDGILGLQLINGKNESAHISDAVGVVAQAVHELLEKENITDPPRGCVGNTNIWKTGPLFKRVLMSSKYADGVTGRVEFNEDGDRKFANYSIMNLQNRKLVQVGIYNGTHVIPNDRKIIWPGGETEKPRGYQMSTRLKIVTIHQEPFVYVKPTLSDGTCKEEFTVNGDPVKKVICTGPNDTSPGSPRHTVPQCCYGFCIDLLIKLARTMNFTYEVHLVADGKFGTQERVNNSNKKEWNGMMGELLSGQADMIVAPLTINNERAQYIEFSKPFKYQGLTILVKKEIPRSTLDSFMQPFQSTLWLLVGLSVHVVAVMLYLLDRFSPFGRFKVNSEEEEEDALTLSSAMWFSWGVLLNSGIGEGAPRSFSARILGMVWAGFAMIIVASYTANLAAFLVLDRPEERITGINDPRLRNPSDKFIYATVKQSSVDIYFRRQVELSTMYRHMEKHNYESAAEAIQAVRDNKLHAFIWDSAVLEFEASQKCDLVTTGELFFRSGFGIGMRKDSPWKQNVSLSILKSHENGFMEDLDKTWVRYQECDSRSNAPATLTFENMAGVFMLVAGGIVAGIFLIFIEIAYKRHKDARRKQMQLAFAAVNVWRKNLQDRKSGRAEPDPKKKATFRAITSTLASSFKRRRSSKDTSTGGGRGALQNQKDTVLPRRAIEREEGQLQLCSRHRES",
# KDSR_HUMAN
"MLLLAAAFLVAFVLLLYMVSPLISPKPLALPGAHVVVTGGSSGIGKCIAIECYKQGAFITLVARNEDKLLQAKKEIEMHSINDKQVVLCISVDVSQDYNQVENVIKQAQEKLGPVDMLVNCAGMAVSGKFEDLEVSTFERLMSINYLGSVYPSRAVITTMKERRVGRIVFVSSQAGQLGLFGFTAYSASKFAIRGLAEALQMEVKPYNVYITVAYPPDTDTPGFAEENRTKPLETRLISETTSVCKPEQVAKQIVKDAIQGNFNSSLGSDGYMLSALTCGMAPVTSITEGLQQVVTMGLFRTIALFYLGSFDSIVRRCMMQREKSENADKTA",
# CKAP4_HUMAN
"MPSAKQRGSKGGHGAASPSEKGAHPSGGADDVAKKPPPAPQQPPPPPAPHPQQHPQQHPQNQAHGKGGHRGGGGGGGKSSSSSSASAAAAAAAASSSASCSRRLGRALNFLFYLALVAAAAFSGWCVHHVLEEVQQVRRSHQDFSRQREELGQGLQGVEQKVQSLQATFGTFESILRSSQHKQDLTEKAVKQGESEVSRISEVLQKLQNEILKDLSDGIHVVKDARERDFTSLENTVEERLTELTKSINDNIAIFTEVQKRSQKEINDMKAKVASLEESEGNKQDLKALKEAVKEIQTSAKSREWDMEALRSTLQTMESDIYTEVRELVSLKQEQQAFKEAADTERLALQALTEKLLRSEESVSRLPEEIRRLEEELRQLKSDSHGPKEDGGFRHSEAFEALQQKSQGLDSRLQHVEDGVLSMQVASARQTESLESLLSKSQEHEQRLAALQGRLEGLGSSEADQDGLASTVRSLGETQLVLYGDVEELKRSVGELPSTVESLQKVQEQVHTLLSQDQAQAARLPPQDFLDRLSSLDNLKASVSQVEADLKMLRTAVDSLVAYSVKIETNENNLESAKGLLDDLRNDLDRLFVKVEKIHEKV",
# BAX_HUMAN
"MDGSGEQPRGGGPTSSEQIMKTGALLLQGFIQDRAGRMGGEAPELALDPVPQDASTKKLSECLKRIGDELDSNMELQRMIAAVDTDSPREVFFRVAADMFSDGNFNWGRVVALFYFASKLVLKALCTKVPELIRTIMGWTLDFLRERLLGWIQDQGGWDGLLSYFGTPTWQTVTIFVAGVLTASLTIWKKMG",
# SCAP_HUMAN
"MTLTERLREKISRAFYNHGLLCASYPIPIILFTGFCILACCYPLLKLPLPGTGPVEFTTPVKDYSPPPVDSDRKQGEPTEQPEWYVGAPVAYVQQIFVKSSVFPWHKNLLAVDVFRSPLSRAFQLVEEIRNHVLRDSSGIRSLEELCLQVTDLLPGLRKLRNLLPEHGCLLLSPGNFWQNDWERFHADPDIIGTIHQHEPKTLQTSATLKDLLFGVPGKYSGVSLYTRKRMVSYTITLVFQHYHAKFLGSLRARLMLLHPSPNCSLRAESLVHVHFKEEIGVAELIPLVTTYIILFAYIYFSTRKIDMVKSKWGLALAAVVTVLSSLLMSVGLCTLFGLTPTLNGGEIFPYLVVVIGLENVLVLTKSVVSTPVDLEVKLRIAQGLSSESWSIMKNMATELGIILIGYFTLVPAIQEFCLFAVVGLVSDFFLQMLFFTTVLSIDIRRMELADLNKRLPPEACLPSAKPVGQPTRYERQLAVRPSTPHTITLQPSSFRNLRLPKRLRVVYFLARTRLAQRLIMAGTVVWIGILVYTDPAGLRNYLAAQVTEQSPLGEGALAPMPVPSGMLPPSHPDPAFSIFPPDAPKLPENQTSPGESPERGGPAEVVHDSPVPEVTWGPEDEELWRKLSFRHWPTLFSYYNITLAKRYISLLPVIPVTLRLNPREALEGRHPQDGRSAWPPPGPIPAGHWEAGPKGPGGVQAHGDVTLYKVAALGLATGIVLVLLLLCLYRVLCPRNYGQLGGGPGRRRRGELPCDDYGYAPPETEIVPLVLRGHLMDIECLASDGMLLVSCCLAGHVCVWDAQTGDCLTRIPRPGRQRRDSGVGSGLEAQESWERLSDGGKAGPEEPGDSPPLRHRPRGPPPPSLFGDQPDLTCLIDTNFSAQPRSSQPTQPEPRHRAVCGRSRDSPGYDFSCLVQRVYQEEGLAAVCTPALRPPSPGPVLSQAPEDEGGSPEKGSPSLAWAPSAEGSIWSLELQGNLIVVGRSSGRLEVWDAIEGVLCCSSEEVSSGITALVFLDKRIVAARLNGSLDFFSLETHTALSPLQFRGTPGRGSSPASPVYSSSDTVACHLTHTVPCAHQKPITALKAAAGRLVTGSQDHTLRVFRLEDSCCLFTLQGHSGAITTVYIDQTMVLASGGQDGAICLWDVLTGSRVSHVFAHRGDVTSLTCTTSCVISSGLDDLISIWDRSTGIKFYSIQQDLGCGASLGVISDNLLVTGGQGCVSFWDLNYGDLLQTVYLGKNSEAQPARQILVLDNAAIVCNFGSELSLVYVPSVLEKLD",
# SRBP2_HUMAN
"MDDSGELGGLETMETLTELGDELTLGDIDEMLQFVSNQVGEFPDLFSEQLCSSFPGSGGSGSSSGSSGSSSSSSNGRGSSSGAVDPSVQRSFTQVTLPSFSPSAASPQAPTLQVKVSPTSVPTTPRATPILQPRPQPQPQPQTQLQQQTVMITPTFSTTPQTRIIQQPLIYQNAATSFQVLQPQVQSLVTSSQVQPVTIQQQVQTVQAQRVLTQTANGTLQTLAPATVQTVAAPQVQQVPVLVQPQIIKTDSLVLTTLKTDGSPVMAAVQNPALTALTTPIQTAALQVPTLVGSSGTILTTMPVMMGQEKVPIKQVPGGVKQLEPPKEGERRTTHNIIEKRYRSSINDKIIELKDLVMGTDAKMHKSGVLRKAIDYIKYLQQVNHKLRQENMVLKLANQKNKLLKGIDLGSLVDNEVDLKIEDFNQNVLLMSPPASDSGSQAGFSPYSIDSEPGSPLLDDAKVKDEPDSPPVALGMVDRSRILLCVLTFLCLSFNPLTSLLQWGGAHDSDQHPHSGSGRSVLSFESGSGGWFDWMMPTLLLWLVNGVIVLSVFVKLLVHGEPVIRPHSRSSVTFWRHRKQADLDLARGDFAAAAGNLQTCLAVLGRALPTSRLDLACSLSWNVIRYSLQKLRLVRWLLKKVFQCRRATPATEAGFEDEAKTSARDAALAYHRLHQLHITGKLPAGSACSDVHMALCAVNLAECAEEKIPPSTLVEIHLTAAMGLKTRCGGKLGFLASYFLSRAQSLCGPEHSAVPDSLRWLCHPLGQKFFMERSWSVKSAAKESLYCAQRNPADPIAQVHQAFCKNLLERAIESLVKPQAKKKAGDQEEESCEFSSALEYLKLLHSFVDSVGVMSPPLSRSSVLKSALGPDIICRWWTSAITVAISWLQGDDAAVRSHFTKVERIPKALEVTESPLVKAIFHACRAMHASLPGKADGQQSSFCHCERASGHLWSSLNVSGATSDPALNHVVQLLTCDLLLSLRTALWQKQASASQAVGETYHASGAELAGFQRDLGSLRRLAHSFRPAYRKVFLHEATVRLMAGASPTRTHQLLEHSLRRRTTQSTKHGEVDAWPGQRERATAILLACRHLPLSFLSSPGQRAVLLAEAARTLEKVGDRRSCNDCQQMIVKLGGGTAIAAS",
# ASPH_HUMAN
"MAQRKNAKSSGNSSSSGSGSGSTSAGSSSPGARRETKHGGHKNGRKGGLSGTSFFTWFMVIALLGVWTSVAVVWFDLVDYEEVLGKLGIYDADGDGDFDVDDAKVLLGLKERSTSEPAVPPEEAEPHTEPEEQVPVEAEPQNIEDEAKEQIQSLLHEMVHAEHVEGEDLQQEDGPTGEPQQEDDEFLMATDVDDRFETLEPEVSHEETEHSYHVEETVSQDCNQDMEEMMSEQENPDSSEPVVEDERLHHDTDDVTYQVYEEQAVYEPLENEGIEITEVTAPPEDNPVEDSQVIVEEVSIFPVEEQQEVPPETNRKTDDPEQKAKVKKKKPKLLNKFDKTIKAELDAAEKLRKRGKIEEAVNAFKELVRKYPQSPRARYGKAQCEDDLAEKRRSNEVLRGAIETYQEVASLPDVPADLLKLSLKRRSDRQQFLGHMRGSLLTLQRLVQLFPNDTSLKNDLGVGYLLIGDNDNAKKVYEEVLSVTPNDGFAKVHYGFILKAQNKIAESIPYLKEGIESGDPGTDDGRFYFHLGDAMQRVGNKEAYKWYELGHKRGHFASVWQRSLYNVNGLKAQPWWTPKETGYTELVKSLERNWKLIRDEGLAVMDKAKGLFLPEDENLREKGDWSQFTLWQQGRRNENACKGAPKTCTLLEKFPETTGCRRGQIKYSIMHPGTHVWPHTGPTNCRLRMHLGLVIPKEGCKIRCANETKTWEEGKVLIFDDSFEHEVWQDASSFRLIFIVDVWHPELTPQQRRSLPAI",
# NMDE1_HUMAN
"MGRVGYWTLLVLPALLVWRGPAPSAAAEKGPPALNIAVMLGHSHDVTERELRTLWGPEQAAGLPLDVNVVALLMNRTDPKSLITHVCDLMSGARIHGLVFGDDTDQEAVAQMLDFISSHTFVPILGIHGGASMIMADKDPTSTFFQFGASIQQQATVMLKIMQDYDWHVFSLVTTIFPGYREFISFVKTTVDNSFVGWDMQNVITLDTSFEDAKTQVQLKKIHSSVILLYCSKDEAVLILSEARSLGLTGYDFFWIVPSLVSGNTELIPKEFPSGLISVSYDDWDYSLEARVRDGIGILTTAASSMLEKFSYIPEAKASCYGQMERPEVPMHTLHPFMVNVTWDGKDLSFTEEGYQVHPRLVVIVLNKDREWEKVGKWENHTLSLRHAVWPRYKSFSDCEPDDNHLSIVTLEEAPFVIVEDIDPLTETCVRNTVPCRKFVKINNSTNEGMNVKKCCKGFCIDILKKLSRTVKFTYDLYLVTNGKHGKKVNNVWNGMIGEVVYQRAVMAVGSLTINEERSEVVDFSVPFVETGISVMVSRSNGTVSPSAFLEPFSASVWVMMFVMLLIVSAIAVFVFEYFSPVGYNRNLAKGKAPHGPSFTIGKAIWLLWGLVFNNSVPVQNPKGTTSKIMVSVWAFFAVIFLASYTANLAAFMIQEEFVDQVTGLSDKKFQRPHDYSPPFRFGTVPNGSTERNIRNNYPYMHQYMTKFNQKGVEDALVSLKTGKLDAFIYDAAVLNYKAGRDEGCKLVTIGSGYIFATTGYGIALQKGSPWKRQIDLALLQFVGDGEMEELETLWLTGICHNEKNEVMSSQLDIDNMAGVFYMLAAAMALSLITFIWEHLFYWKLRFCFTGVCSDRPGLLFSISRGIYSCIHGVHIEEKKKSPDFNLTGSQSNMLKLLRSAKNISSMSNMNSSRMDSPKRAADFIQRGSLIMDMVSDKGNLMYSDNRSFQGKESIFGDNMNELQTFVANRQKDNLNNYVFQGQHPLTLNESNPNTVEVAVSTESKANSRPRQLWKKSVDSIRQDSLSQNPVSQRDEATAENRTHSLKSPRYLPEEMAHSDISETSNRATCHREPDNSKNHKTKDNFKRSVASKYPKDCSEVERTYLKTKSSSPRDKIYTIDGEKEPGFHLDPPQFVENVTLPENVDFPDPYQDPSENFRKGDSTLPMNRNPLHNEEGLSNNDQYKLYSKHFTLKDKGSPHSETSERYRQNSTHCRSCLSNMPTYSGHFTMRSPFKCDACLRMGNLYDIDEDQMLQETGNPATGEQVYQQDWAQNNALQLQKNKLRISRQHSYDNIVDKPRELDLSRPSRSISLKDRERLLEGNFYGSLFSVPSSKLSGKKSSLFPQGLEDSKRSKSLLPDHTSDNPFLHSHRDDQRLVIGRCPSDPYKHSLPSQAVNDSYLRSSLRSTASYCSRDSRGHNDVYISEHVMPYAANKNNMYSTPRVLNSCSNRRVYKKMPSIESDV",
# DLG1_HUMAN
"MPVRKQDTQRALHLLEEYRSKLSQTEDRQLRSSIERVINIFQSNLFQALIDIQEFYEVTLLDNPKCIDRSKPSEPIQPVNTWEISSLPSSTVTSETLPSSLSPSVEKYRYQDEDTPPQEHISPQITNEVIGPELVHVSEKNLSEIENVHGFVSHSHISPIKPTEAVLPSPPTVPVIPVLPVPAENTVILPTIPQANPPPVLVNTDSLETPTYVNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRLRVNDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTALKNTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSPSSFLGQTPASPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKIHDLREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVINASDDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKTRDKGEIPDDMGSKGLKHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPDKFGSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVREVAEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERAMKLEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL",
# SEC20_HUMAN
"MAAPQDVHVRICNQEIVKFDLEVKALIQDIRDCSGPLSALTELNTKVKEKFQQLRHRIQDLEQLAKEQDKESEKQLLLQEVENHKKQMLSNQASWRKANLTCKIAIDNLEKAELLQGGDLLRQRKTTKESLAQTSSTITESLMGISRMMAQQVQQSEEAMQSLVTSSRTILDANEEFKSMSGTIQLGRKLITKYNRRELTDKLLIFLALALFLATVLYIVKKRLFPFL",
# TRDN_HUMAN
"MTEITAEGNASTTTTVIDSKNGSVPKSPGKVLKRTVTEDIVTTFSSPAAWLLVIALIITWSAVAIVMFDLVDYKNFSASSIAKIGSDPLKLVRDAMEETTDWIYGFFSLLSDIISSEDEEDDDGDEDTDKGEIDEPPLRKKEIHKDKTEKQEKPERKIQTKVTHKEKEKGKEKVREKEKPEKKATHKEKIEKKEKPETKTLAKEQKKAKTAEKSEEKTKKEVKGGKQEKVKQTAAKVKEVQKTPSKPKEKEDKEKAAVSKHEQKDQYAFCRYMIDIFVHGDLKPGQSPAIPPPLPTEQASRPTPASPALEEKEGEKKKAEKKVTSETKKKEKEDIKKKSEKETAIDVEKKEPGKASETKQGTVKIAAQAAAKKDEKKEDSKKTKKPAEVEQPKGKKQEKKEKHVEPAKSPKKEHSVPSDKQVKAKTERAKEEIGAVSIKKAVPGKKEEKTTKTVEQEIRKEKSGKTSSILKDKEPIKGKEEKVPASLKEKEPETKKDEKMSKAGKEVKPKPPQLQGKKEEKPEPQIKKEAKPAISEKVQIHKQDIVKPEKTVSHGKPEEKVLKQVKAVTIEKTAKPKPTKKAEHREREPPSIKTDKPKPTPKGTSEVTESGKKKTEISEKESKEKADMKHLREEKVSTRKESLQLHNVTKAEKPARVSKDVEDVPASKKAKEGTEDVSPTKQKSPISFFQCVYLDGYNGYGFQFPFTPADRPGESSGQANSPGQKQQGQ",
# CLN3_HUMAN
"MGGCAGSRRRFSDSEGEETVPEPRLPLLDHQGAHWKNAVGFWLLGLCNNFSYVVMLSAAHDILSHKRTSGNQSHVDPGPTPIPHNSSSRFDCNSVSTAAVLLADILPTLVIKLLAPLGLHLLPYSPRVLVSGICAAGSFVLVAFSHSVGTSLCGVVFASISSGLGEVTFLSLTAFYPRAVISWWSSGTGGAGLLGALSYLGLTQAGLSPQQTLLSMLGIPALLLASYFLLLTSPEAQDPGGEEEAESAARQPLIRTEAPESKPGSSSSLSLRERWTVFKGLLWYIVPLVVVYFAEYFINQGLFELLFFWNTSLSHAQQYRWYQMLYQAGVFASRSSLRCCRIRFTWALALLQCLNLVFLLADVWFGFLPSIYLVFLIILYEGLLGGAAYVNTFHNIALETSDEHREFAMAATCISDTLGISLSGLLALPLHDFLCQLS",
# TMED1_HUMAN
"MMAAGAALALALWLLMPPVEVGGAGPPPIQDGEFTFLLPAGRKQCFYQSAPANASLETEYQVIGGAGLDVDFTLESPQGVLLVSESRKADGVHTVEPTEAGDYKLCFDNSFSTISEKLVFFELIFDSLQDDEEVEGWAEAVEPEEMLDVKMEDIKESIETMRTRLERSIQMLTLLRAFEARDRNLQEGNLERVNFWSAVNVAVLLLVAVLQVCTLKRFFQDKRPVPT",
# PKD2_HUMAN
"MVNSSRVQPQQPGDAKRPPAPRAPDPGRLMAGCAAVGASLAAPGGLCEQRGLEIEMQRIRQAAARDPPAGAAASPSPPLSSCSRQAWSRDNPGFEAEEEEEEVEGEEGGMVVEMDVEWRPGSRRSAASSAVSSVGARSRGLGGYHGAGHPSGRRRRREDQGPPCPSPVGGGDPLHRHLPLEGQPPRVAWAERLVRGLRGLWGTRLMEESSTNREKYLKSVLRELVTYLLFLIVLCILTYGMMSSNVYYYTRMMSQLFLDTPVSKTEKTNFKTLSSMEDFWKFTEGSLLDGLYWKMQPSNQTEADNRSFIFYENLLLGVPRIRQLRVRNGSCSIPQDLRDEIKECYDVYSVSSEDRAPFGPRNGTAWIYTSEKDLNGSSHWGIIATYSGAGYYLDLSRTREETAAQVASLKKNVWLDRGTRATFIDFSVYNANINLFCVVRLLVEFPATGGVIPSWQFQPLKLIRYVTTFDFFLAACEIIFCFFIFYYVVEEILEIRIHKLHYFRSFWNCLDVVIVVLSVVAIGINIYRTSNVEVLLQFLEDQNTFPNFEHLAYWQIQFNNIAAVTVFFVWIKLFKFINFNRTMSQLSTTMSRCAKDLFGFAIMFFIIFLAYAQLAYLVFGTQVDDFSTFQECIFTQFRIILGDINFAEIEEANRVLGPIYFTTFVFFMFFILLNMFLAIINDTYSEVKSDLAQQKAEMELSDLIRKGYHKALVKLKLKKNTVDDISESLRQGGGKLNFDELRQDLKGKGHTDAEIEAIFTKYDQDGDQELTEHEHQQMRDDLEKEREDLDLDHSSLPRPMSSRSFPRSLDDSEEDDDEDSGHSSRRRGSISSGVSYEEFQVLVRRVDRMEHSIGSIVSKIDAVIVKLEIMERAKLKRREVLGRLLDGVAEDERLGRDSEIHREQMERLVREELERWESDDAASQISHGLGTPVGLNGQPRPRSSRPSSSQSTEGMEGAGGNGSSNVHV",
# STIM1_HUMAN
"MDVCVRLALWLLWGLLLHQGQSLSHSHSEKATGTSSGANSEESTAAEFCRIDKPLCHSEDEKLSFEAVRNIHKLMDDDANGDVDVEESDEFLREDLNYHDPTVKHSTFHGEDKLISVEDLWKAWKSSEVYNWTVDEVVQWLITYVELPQYEETFRKLQLSGHAMPRLAVTNTTMTGTVLKMTDRSHRQKLQLKALDTVLFGPPLLTRHNHLKDFMLVVSIVIGVGGCWFAYIQNRYSKEHMKKMMKDLEGLHRAEQSLHDLQERLHKAQEEHRTVEVEKVHLEKKLRDEINLAKQEAQRLKELREGTENERSRQKYAEEELEQVREALRKAEKELESHSSWYAPEALQKWLQLTHEVEVQYYNIKKQNAEKQLLVAKEGAEKIKKKRNTLFGTFHVAHSSSLDDVDHKILTAKQALSEVTAALRERLHRWQQIEILCGFQIVNNPGIHSLVAALNIDPSWMGSTRPNPAHFIMTDDVDDMDEEIVSPLSMQSPSLQSSVRQRLTEPQHGLGSQRDLTHSDSESSLHMSDRQRVAPKPPQMSRAADEALNAMTSNGSHRLIEGVHPGSLVEKLPDSPALAKKALLALNHGLDKAHSLMELSPSAPPGGSPHLDSSRSHSPSSPDPDTPSPVGDSRALQASRNTRIPHLAGKKAVAEEDNGSIGEETDSSPGRKKFPLKIFKKPLKK",
# MOGS_HUMAN
"MARGERRRRAVPAEGVRTAERAARGGPGRRDGRGGGPRSTAGGVALAVVVLSLALGMSGRWVLAWYRARRAVTLHSAPPVLPADSSSPAVAPDLFWGTYRPHVYFGMKTRSPKPLLTGLMWAQQGTTPGTPKLRHTCEQGDGVGPYGWEFHDGLSFGRQHIQDGALRLTTEFVKRPGGQHGGDWSWRVTVEPQDSGTSALPLVSLFFYVVTDGKEVLLPEVGAKGQLKFISGHTSELGDFRFTLLPPTSPGDTAPKYGSYNVFWTSNPGLPLLTEMVKSRLNSWFQHRPPGAPPERYLGLPGSLKWEDRGPSGQGQGQFLIQQVTLKIPISIEFVFESGSAQAGGNQALPRLAGSLLTQALESHAEGFRERFEKTFQLKEKGLSSGEQVLGQAALSGLLGGIGYFYGQGLVLPDIGVEGSEQKVDPALFPPVPLFTAVPSRSFFPRGFLWDEGFHQLVVQRWDPSLTREALGHWLGLLNADGWIGREQILGDEARARVPPEFLVQRAVHANPPTLLLPVAHMLEVGDPDDLAFLRKALPRLHAWFSWLHQSQAGPLPLSYRWRGRDPALPTLLNPKTLPSGLDDYPRASHPSVTERHLDLRCWVALGARVLTRLAEHLGEAEVAAELGPLAASLEAAESLDELHWAPELGVFADFGNHTKAVQLKPRPPQGLVRVVGRPQPQLQYVDALGYVSLFPLLLRLLDPTSSRLGPLLDILADSRHLWSPFGLRSLAASSSFYGQRNSEHDPPYWRGAVWLNVNYLALGALHHYGHLEGPHQARAAKLHGELRANVVGNVWRQYQATGFLWEQYSDRDGRGMGCRPFHGWTSLVLLAMAEDY",
# MLEC_HUMAN
"MLGAWAVEGTAVALLRLLLLLLPPAIRGPGLGVAGVAGAAGAGLPESVIWAVNAGGEAHVDVHGIHFRKDPLEGRVGRASDYGMKLPILRSNPEDQILYQTERYNEETFGYEVPIKEEGDYVLVLKFAEVYFAQSQQKVFDVRLNGHVVVKDLDIFDRVGHSTAHDEIIPMSIRKGKLSVQGEVSTFTGKLYIEFVKGYYDNPKVCALYIMAGTVDDVPKLQPHPGLEKKEEEEEEEEYDEGSNLKKQTNKNRVQSGPRTPNPYASDNSSLMFPILVAFGVFIPTLFCLCRL",
# FKBP8_HUMAN
"MASCAEPSEPSAPLPAGVPPLEDFEVLDGVEDAEGEEEEEEEEEEEDDLSELPPLEDMGQPPAEEAEQPGALAREFLAAMEPEPAPAPAPEEWLDILGNGLLRKKTLVPGPPGSSRPVKGQVVTVHLQTSLENGTRVQEEPELVFTLGDCDVIQALDLSVPLMDVGETAMVTADSKYCYGPQGRSPYIPPHAALCLEVTLKTAVDGPDLEMLTGQERVALANRKRECGNAHYQRADFVLAANSYDLAIKAITSSAKVDMTFEEEAQLLQLKVKCLNNLAASQLKLDHYRAALRSCSLVLEHQPDNIKALFRKGKVLAQQGEYSEAIPILRAALKLEPSNKTIHAELSKLVKKHAAQRSTETALYRKMLGNPSRLPAKCPGKGAWSIPWKWLFGATAVALGGVALSVVIAARN",
# PIGH_HUMAN
"MEDERSFSDICGGRLALQRRYYSPSCREFCLSCPRLSLRSLTAVTCTVWLAAYGLFTLCENSMILSAAIFITLLGLLGYLHFVKIDQETLLIIDSLGIQMTSSYASGKESTTFIEMGKVKDIVINEAIYMQKVIYYLCILLKDPVEPHGISQVVPVFQSAKPRLDCLIEVYRSCQEILAHQKATSTSP",
# BECN1_HUMAN
"MEGSKTSNNSTMQVSFVCQRCSQPLKLDTSFKILDRVTIQELTAPLLTTAQAKPGETQEEETNSGEEPFIETPRQDGVSRRFIPPARMMSTESANSFTLIGEASDGGTMENLSRRLKVTGDLFDIMSGQTDVDHPLCEECTDTLLDQLDTQLNVTENECQNYKRCLEILEQMNEDDSEQLQMELKELALEEERLIQELEDVEKNRKIVAENLEKVQAEAERLDQEEAQYQREYSEFKRQQLELDDELKSVENQMRYAQTQLDKLKKTNVFNATFHIWHSGQFGTINNFRLGRLPSVPVEWNEINAAWGQTVLLLHALANKMGLKFQRYRLVPYGNHSYLESLTDKSKELPLYCSGGLRFFWDNKFDHAMVAFLDCVQQFKEEVEKGETRFCLPYRMDVEKGKIEDTGGSGGSYSIKTQFNSEEQWTKALKFMLTNLKWGLAWVSSQFYNK",
# ERG1_HUMAN
"MWTFLGIATFTYFYKKFGDFITLANREVLLCVLVFLSLGLVLSYRCRHRNGGLLGRQQSGSQFALFSDILSGLPFIGFFWAKSPPESENKEQLEARRRRKGTNISETSLIGTAACTSTSSQNDPEVIIVGAGVLGSALAAVLSRDGRKVTVIERDLKEPDRIVGEFLQPGGYHVLKDLGLGDTVEGLDAQVVNGYMIHDQESKSEVQIPYPLSENNQVQSGRAFHHGRFIMSLRKAAMAEPNAKFIEGVVLQLLEEDDVVMGVQYKDKETGDIKELHAPLTVVADGLFSKFRKSLVSNKVSVSSHFVGFLMKNAPQFKANHAELILANPSPVLIYQISSSETRVLVDIRGEMPRNLREYMVEKIYPQIPDHLKEPFLEATDNSHLRSMPASFLPPSSVKKRGVLLLGDAYNMRHPLTGGGMTVAFKDIKLWRKLLKGIPDLYDDAAIFEAKKSFYWARKTSHSFVVNILAQALYELFSATDDSLHQLRKACFLYFKLGGECVAGPVGLLSVLSPNPLVLIGHFFAVAIYAVYFCFKSEPWITKPRALLSSGAVLYKACSVIFPLIYSEMKYMVH",
# ITPR2_HUMAN
"MTEKMSSFLYIGDIVSLYAEGSVNGFISTLGLVDDRCVVHPEAGDLANPPKKFRDCLFKVCPMNRYSAQKQYWKAKQAKQGNHTEAALLKKLQHAAELEQKQNESENKKLLGEIVKYSNVIQLLHIKSNKYLTVNKRLPALLEKNAMRVSLDAAGNEGSWFYIHPFWKLRSEGDNIVVGDKVVLMPVNAGQPLHASNIELLDNPGCKEVNAVNCNTSWKITLFMKYSSYREDVLKGGDVVRLFHAEQEKFLTCDEYEKKQHIFLRTTLRQSATSATSSKALWEIEVVHHDPCRGGAGQWNSLFRFKHLATGNYLAAELNPDYRDAQNEGKNVRDGVPPTSKKKRQAGEKIMYTLVSVPHGNDIASLFELDATTLQRADCLVPRNSYVRLRHLCTNTWVTSTSIPIDTDEERPVMLKIGTCQTKEDKEAFAIVSVPLSEVRDLDFANDANKVLATTVKKLENGTITQNERRFVTKLLEDLIFFVADVPNNGQEVLDVVITKPNRERQKLMREQNILAQVFGILKAPFKEKAGEGSMLRLEDLGDQRYAPYKYMLRLCYRVLRHSQQDYRKNQEYIAKNFCVMQSQIGYDILAEDTITALLHNNRKLLEKHITAKEIETFVSLLRRNREPRFLDYLSDLCVSNTTAIPVTQELICKFMLSPGNADILIQTKVVSMQADNPMESSILSDDIDDEEVWLYWIDSNKEPHGKAIRHLAQEAKEGTKADLEVLTYYRYQLNLFARMCLDRQYLAINQISTQLSVDLILRCVSDESLPFDLRASFCRLMLHMHVDRDPQESVVPVRYARLWTEIPTKITIHEYDSITDSSRNDMKRKFALTMEFVEEYLKEVVNQPFPFGDKEKNKLTFEVVHLARNLIYFGFYSFSELLRLTRTLLAILDIVQAPMSSYFERLSKFQDGGNNVMRTIHGVGEMMTQMVLSRGSIFPMSVPDVPPSIHPSKQGSPTEHEDVTVMDTKLKIIEILQFILSVRLDYRISYMLSIYKKEFGEDNDNAETSASGSPDTLLPSAIVPDIDEIAAQAETMFAGRKEKNPVQLDDEGGRTFLRVLIHLIMHDYPPLLSGALQLLFKHFSQRAEVLQAFKQVQLLVSNQDVDNYKQIKADLDQLRLTVEKSELWVEKSSNYENGEIGESQVKGGEEPIEESNILSPVQDGTKKPQIDSNKSNNYRIVKEILIRLSKLCVQNKKCRNQHQRLLKNMGAHSVVLDLLQIPYEKNDEKMNEVMNLAHTFLQNFCRGNPQNQVLLHKHLNLFLTPGLLEAETMRHIFMNNYHLCNEISERVVQHFVHCIETHGRHVEYLRFLQTIVKADGKYVKKCQDMVMTELINGGEDVLIFYNDRASFPILLHMMCSERDRGDESGPLAYHITLVELLAACTEGKNVYTEIKCNSLLPLDDIVRVVTHDDCIPEVKIAYVNFVNHCYVDTEVEMKEIYTSNHIWKLFENFLVDMARVCNTTTDRKHADIFLEKCVTESIMNIVSGFFNSPFSDNSTSLQTHQPVFIQLLQSAFRIYNCTWPNPAQKASVESCIRTLAEVAKNRGIAIPVDLDSQVNTLFMKSHSNMVQRAAMGWRLSARSGPRFKEALGGPAWDYRNIIEKLQDVVASLEHQFSPMMQAEFSVLVDVLYSPELLFPEGSDARIRCGAFMSKLINHTKKLMEKEEKLCIKILQTLREMLEKKDSFVEEGNTLRKILLNRYFKGDYSIGVNGHLSGAYSKTAQVGGSFSGQDSDKMGISMSDIQCLLDKEGASELVIDVIVNTKNDRIFSEGIFLGIALLEGGNTQTQYSFYQQLHEQKKSEKFFKVLYDRMKAAQKEIRSTVTVNTIDLGNKKRDDDNELMTSGPRMRVRDSTLHLKEGMKGQLTEASSATSKAYCVYRREMDPEIDIMCTGPEAGNTEEKSAEEVTMSPAIAIMQPILRFLQLLCENHNRELQNFLRNQNNKTNYNLVCETLQFLDCICGSTTGGLGLLGLYINEKNVALVNQNLESLTEYCQGPCHENQTCIATHESNGIDIIIALILNDINPLGKYRMDLVLQLKNNASKLLLAIMESRHDSENAERILFNMRPRELVDVMKNAYNQGLECDHGDDEGGDDGVSPKDVGHNIYILAHQLARHNKLLQQMLKPGSDPDEGDEALKYYANHTAQIEIVRHDRTMEQIVFPVPNICEYLTRESKCRVFNTTERDEQGSKVNDFFQQTEDLYNEMKWQKKIRNNPALFWFSRHISLWGSISFNLAVFINLAVALFYPFGDDGDEGTLSPLFSVLLWIAVAICTSMLFFFSKPVGIRPFLVSIMLRSIYTIGLGPTLILLGAANLCNKIVFLVSFVGNRGTFTRGYRAVILDMAFLYHVAYVLVCMLGLFVHEFFYSFLLFDLVYREETLLNVIKSVTRNGRSIILTAVLALILVYLFSIIGFLFLKDDFTMEVDRLKNRTPVTGSHQVPTMTLTTMMEACAKENCSPTIPASNTADEEYEDGIERTCDTLLMCIVTVLNQGLRNGGGVGDVLRRPSKDEPLFAARVVYDLLFYFIVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVSFEEHIKSEHNMWHYLYFIVLVKVKDPTEYTGPESYVAQMIVEKNLDWFPRMRAMSLVSNEGDSEQNEIRSLQEKLESTMSLVKQLSGQLAELKEQMTEQRKNKQRLGFLGSNTPHVNHHMPPH",
# ITPR3_HUMAN
"MSEMSSFLHIGDIVSLYAEGSVNGFISTLGLVDDRCVVEPAAGDLDNPPKKFRDCLFKVCPMNRYSAQKQYWKAKQTKQDKEKIADVVLLQKLQHAAQMEQKQNDTENKKVHGDVVKYGSVIQLLHMKSNKYLTVNKRLPALLEKNAMRVTLDATGNEGSWLFIQPFWKLRSNGDNVVVGDKVILNPVNAGQPLHASNYELSDNAGCKEVNSVNCNTSWKINLFMQFRDHLEEVLKGGDVVRLFHAEQEKFLTCDEYKGKLQVFLRTTLRQSATSATSSNALWEVEVVHHDPCRGGAGHWNGLYRFKHLATGNYLAAEENPSYKGDASDPKAAGMGAQGRTGRRNAGEKIKYCLVAVPHGNDIASLFELDPTTLQKTDSFVPRNSYVRLRHLCTNTWIQSTNVPIDIEEERPIRLMLGTCPTKEDKEAFAIVSVPVSEIRDLDFANDASSMLASAVEKLNEGFISQNDRRFVIQLLEDLVFFVSDVPNNGQNVLDIMVTKPNRERQKLMREQNILKQVFGILKAPFREKGGEGPLVRLEELSDQKNAPYQHMFRLCYRVLRHSQEDYRKNQEHIAKQFGMMQSQIGYDILAEDTITALLHNNRKLLEKHITKTEVETFVSLVRKNREPRFLDYLSDLCVSNHIAIPVTQELICKCVLDPKNSDILIRTELRPVKEMAQSHEYLSIEYSEEEVWLTWTDKNNEHHEKSVRQLAQEARAGNAHDENVLSYYRYQLKLFARMCLDRQYLAIDEISQQLGVDLIFLCMADEMLPFDLRASFCHLMLHVHVDRDPQELVTPVKFARLWTEIPTAITIKDYDSNLNASRDDKKNKFANTMEFVEDYLNNVVSEAVPFANEEKNKLTFEVVSLAHNLIYFGFYSFSELLRLTRTLLGIIDCVQGPPAMLQAYEDPGGKNVRRSIQGVGHMMSTMVLSRKQSVFSAPSLSAGASAAEPLDRSKFEENEDIVVMETKLKILEILQFILNVRLDYRISYLLSVFKKEFVEVFPMQDSGADGTAPAFDSTTANMNLDRIGEQAEAMFGVGKTSSMLEVDDEGGRMFLRVLIHLTMHDYAPLVSGALQLLFKHFSQRQEAMHTFKQVQLLISAQDVENYKVIKSELDRLRTMVEKSELWVDKKGSGKGEEVEAGAAKDKKERPTDEEGFLHPPGEKSSENYQIVKGILERLNKMCGVGEQMRKKQQRLLKNMDAHKVMLDLLQIPYDKGDAKMMEILRYTHQFLQKFCAGNPGNQALLHKHLHLFLTPGLLEAETMQHIFLNNYQLCSEISEPVLQHFVHLLATHGRHVQYLDFLHTVIKAEGKYVKKCQDMIMTELTNAGDDVVVFYNDKASLAHLLDMMKAARDGVEDHSPLMYHISLVDLLAACAEGKNVYTEIKCTSLLPLEDVVSVVTHEDCITEVKMAYVNFVNHCYVDTEVEMKEIYTSNHIWTLFENFTLDMARVCSKREKRVADPTLEKYVLSVVLDTINAFFSSPFSENSTSLQTHQTIVVQLLQSTTRLLECPWLQQQHKGSVEACIRTLAMVAKGRAILLPMDLDAHISSMLSSGASCAAAAQRNASSYKATTRAFPRVTPTANQWDYKNIIEKLQDIITALEERLKPLVQAELSVLVDVLHWPELLFLEGSEAYQRCESGGFLSKLIQHTKDLMESEEKLCIKVLRTLQQMLLKKTKYGDRGNQLRKMLLQNYLQNRKSTSRGDLPDPIGTGLDPDWSAIAATQCRLDKEGATKLVCDLITSTKNEKIFQESIGLAIHLLDGGNTEIQKSFHNLMMSDKKSERFFKVLHDRMKRAQQETKSTVAVNMNDLGSQPHEDREPVDPTTKGRVASFSIPGSSSRYSLGPSLRRGHEVSERVQSSEMGTSVLIMQPILRFLQLLCENHNRDLQNFLRCQNNKTNYNLVCETLQFLDIMCGSTTGGLGLLGLYINEDNVGLVIQTLETLTEYCQGPCHENQTCIVTHESNGIDIITALILNDISPLCKYRMDLVLQLKDNASKLLLALMESRHDSENAERILISLRPQELVDVIKKAYLQEEERENSEVSPREVGHNIYILALQLSRHNKQLQHLLKPVKRIQEEEAEGISSMLSLNNKQLSQMLKSSAPAQEEEEDPLAYYENHTSQIEIVRQDRSMEQIVFPVPGICQFLTEETKHRLFTTTEQDEQGSKVSDFFDQSSFLHNEMEWQRKLRSMPLIYWFSRRMTLWGSISFNLAVFINIIIAFFYPYMEGASTGVLDSPLISLLFWILICFSIAALFTKRYSIRPLIVALILRSIYYLGIGPTLNILGALNLTNKIVFVVSFVGNRGTFIRGYKAMVMDMEFLYHVGYILTSVLGLFAHELFYSILLFDLIYREETLFNVIKSVTRNGRSILLTALLALILVYLFSIVGFLFLKDDFILEVDRLPNNHSTASPLGMPHGAAAFVDTCSGDKMDCVSGLSVPEVLEEDRELDSTERACDTLLMCIVTVMNHGLRNGGGVGDILRKPSKDESLFPARVVYDLLFFFIVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVSFEEHIKLEHNMWNYLYFIVLVRVKNKTDYTGPESYVAQMIKNKNLDWFPRMRAMSLVSNEGEGEQNEIRILQDKLNSTMKLVSHLTAQLNELKEQMTEQRKRRQRLGFVDVQNCISR",
# ITPR1_HUMAN
"MSDKMSSFLHIGDICSLYAEGSTNGFISTLGLVDDRCVVQPETGDLNNPPKKFRDCLFKLCPMNRYSAQKQFWKAAKPGANSTTDAVLLNKLHHAADLEKKQNETENRKLLGTVIQYGNVIQLLHLKSNKYLTVNKRLPALLEKNAMRVTLDEAGNEGSWFYIQPFYKLRSIGDSVVIGDKVVLNPVNAGQPLHASSHQLVDNPGCNEVNSVNCNTSWKIVLFMKWSDNKDDILKGGDVVRLFHAEQEKFLTCDEHRKKQHVFLRTTGRQSATSATSSKALWEVEVVQHDPCRGGAGYWNSLFRFKHLATGHYLAAEVDPDFEEECLEFQPSVDPDQDASRSRLRNAQEKMVYSLVSVPEGNDISSIFELDPTTLRGGDSLVPRNSYVRLRHLCTNTWVHSTNIPIDKEEEKPVMLKIGTSPVKEDKEAFAIVPVSPAEVRDLDFANDASKVLGSIAGKLEKGTITQNERRSVTKLLEDLVYFVTGGTNSGQDVLEVVFSKPNRERQKLMREQNILKQIFKLLQAPFTDCGDGPMLRLEELGDQRHAPFRHICRLCYRVLRHSQQDYRKNQEYIAKQFGFMQKQIGYDVLAEDTITALLHNNRKLLEKHITAAEIDTFVSLVRKNREPRFLDYLSDLCVSMNKSIPVTQELICKAVLNPTNADILIETKLVLSRFEFEGVSSTGENALEAGEDEEEVWLFWRDSNKEIRSKSVRELAQDAKEGQKEDRDVLSYYRYQLNLFARMCLDRQYLAINEISGQLDVDLILRCMSDENLPYDLRASFCRLMLHMHVDRDPQEQVTPVKYARLWSEIPSEIAIDDYDSSGASKDEIKERFAQTMEFVEEYLRDVVCQRFPFSDKEKNKLTFEVVNLARNLIYFGFYNFSDLLRLTKILLAILDCVHVTTIFPISKMAKGEENKGNNDVEKLKSSNVMRSIHGVGELMTQVVLRGGGFLPMTPMAAAPEGNVKQAEPEKEDIMVMDTKLKIIEILQFILNVRLDYRISCLLCIFKREFDESNSQTSETSSGNSSQEGPSNVPGALDFEHIEEQAEGIFGGSEENTPLDLDDHGGRTFLRVLLHLTMHDYPPLVSGALQLLFRHFSQRQEVLQAFKQVQLLVTSQDVDNYKQIKQDLDQLRSIVEKSELWVYKGQGPDETMDGASGENEHKKTEEGNNKPQKHESTSSYNYRVVKEILIRLSKLCVQESASVRKSRKQQQRLLRNMGAHAVVLELLQIPYEKAEDTKMQEIMRLAHEFLQNFCAGNQQNQALLHKHINLFLNPGILEAVTMQHIFMNNFQLCSEINERVVQHFVHCIETHGRNVQYIKFLQTIVKAEGKFIKKCQDMVMAELVNSGEDVLVFYNDRASFQTLIQMMRSERDRMDENSPLMYHIHLVELLAVCTEGKNVYTEIKCNSLLPLDDIVRVVTHEDCIPEVKIAYINFLNHCYVDTEVEMKEIYTSNHMWKLFENFLVDICRACNNTSDRKHADSILEKYVTEIVMSIVTTFFSSPFSDQSTTLQTRQPVFVQLLQGVFRVYHCNWLMPSQKASVESCIRVLSDVAKSRAIAIPVDLDSQVNNLFLKSHSIVQKTAMNWRLSARNAARRDSVLAASRDYRNIIERLQDIVSALEDRLRPLVQAELSVLVDVLHRPELLFPENTDARRKCESGGFICKLIKHTKQLLEENEEKLCIKVLQTLREMMTKDRGYGEKLISIDELDNAELPPAPDSENATEELEPSPPLRQLEDHKRGEALRQVLVNRYYGNVRPSGRRESLTSFGNGPLSAGGPGKPGGGGGGSGSSSMSRGEMSLAEVQCHLDKEGASNLVIDLIMNASSDRVFHESILLAIALLEGGNTTIQHSFFCRLTEDKKSEKFFKVFYDRMKVAQQEIKATVTVNTSDLGNKKKDDEVDRDAPSRKKAKEPTTQITEEVRDQLLEASAATRKAFTTFRREADPDDHYQPGEGTQATADKAKDDLEMSAVITIMQPILRFLQLLCENHNRDLQNFLRCQNNKTNYNLVCETLQFLDCICGSTTGGLGLLGLYINEKNVALINQTLESLTEYCQGPCHENQNCIATHESNGIDIITALILNDINPLGKKRMDLVLELKNNASKLLLAIMESRHDSENAERILYNMRPKELVEVIKKAYMQGEVEFEDGENGEDGAASPRNVGHNIYILAHQLARHNKELQSMLKPGGQVDGDEALEFYAKHTAQIEIVRLDRTMEQIVFPVPSICEFLTKESKLRIYYTTERDEQGSKINDFFLRSEDLFNEMNWQKKLRAQPVLYWCARNMSFWSSISFNLAVLMNLLVAFFYPFKGVRGGTLEPHWSGLLWTAMLISLAIVIALPKPHGIRALIASTILRLIFSVGLQPTLFLLGAFNVCNKIIFLMSFVGNCGTFTRGYRAMVLDVEFLYHLLYLVICAMGLFVHEFFYSLLLFDLVYREETLLNVIKSVTRNGRSIILTAVLALILVYLFSIVGYLFFKDDFILEVDRLPNETAVPETGESLASEFLFSDVCRVESGENCSSPAPREELVPAEETEQDKEHTCETLLMCIVTVLSHGLRSGGGVGDVLRKPSKEEPLFAARVIYDLLFFFMVIIIVLNLIFGVIIDTFADLRSEKQKKEEILKTTCFICGLERDKFDNKTVTFEEHIKEEHNMWHYLCFIVLVKVKDSTEYTGPESYVAEMIKERNLDWFPRMRAMSLVSSDSEGEQNELRNLQEKLESTMKLVTNLSGQLSELKDQMTEQRKQKQRIGLLGHPPHMNVNPQQPA",
# EMC2_HUMAN
"MAKVSELYDVTWEEMRDKMRKWREENSRNSEQIVEVGEELINEYASKLGDDIWIIYEQVMIAALDYGRDDLALFCLQELRRQFPGSHRVKRLTGMRFEAMERYDDAIQLYDRILQEDPTNTAARKRKIAIRKAQGKNVEAIRELNEYLEQFVGDQEAWHELAELYINEHDYAKAAFCLEELMMTNPHNHLYCQQYAEVKYTQGGLENLELSRKYFAQALKLNNRNMRALFGLYMSASHIASNPKASAKTKKDNMKYASWAASQINRAYQFAGRSKKETKYSLKAVEDMLETLQITQS",
# HERP1_HUMAN
"MESETEPEPVTLLVKSPNQRHRDLELSGDRGWSVGHLKAHLSRVYPERPRPEDQRLIYSGKLLLDHQCLRDLLPKQEKRHVLHLVCNVKSPSKMPEINAKVAESTEEPAGSNRGQYPEDSSSDGLRQREVLRNLSSPGWENISRPEAAQQAFQGLGPGFSGYTPYGWLQLSWFQQIYARQYYMQYLAATAASGAFVPPPSAQEIPVVSAPAPAPIHNQFPAENQPANQNAAPQVVVNPGANQNLRMNAQGGPIVEEDDEINRDWLDWTYSAATFSVFLSILYFYSSLSRFLMVMGATVVMYLHHVGWFPFRPRPVQNFPNDGPPPDVVNQDPNNNLQEGTDPETEDPNHLPPDRDVLDGEQTSPSFMSTAWLVFKTFFASLLPEGPPAIAN",
# PDIA6_HUMAN
"MALLVLGLVSCTFFLAVNGLYSSSDDVIELTPSNFNREVIQSDSLWLVEFYAPWCGHCQRLTPEWKKAATALKDVVKVGAVDADKHHSLGGQYGVQGFPTIKIFGSNKNRPEDYQGGRTGEAIVDAALSALRQLVKDRLGGRSGGYSSGKQGRSDSSSKKDVIELTDDSFDKNVLDSEDVWMVEFYAPWCGHCKNLEPEWAAAASEVKEQTKGKVKLAAVDATVNQVLASRYGIRGFPTIKIFQKGESPVDYDGGRTRSDIVSRALDLFSDNAPPPELLEIINEDIAKRTCEEHQLCVVAVLPHILDTGAAGRNSYLEVLLKLADKYKKKMWGWLWTEAGAQSELETALGIGGFGYPAMAAINARKMKFALLKGSFSEQGINEFLRELSFGRGSTAPVGGGAFPTIVEREPWDGRDGELPVEDDIDLSDVELDDLGKDEL",
# EBP_HUMAN
"MTTNAGPLHPYWPQHLRLDNFVPNDRPTWHILAGLFSVTGVLVVTTWLLSGRAAVVPLGTWRRLSLCWFAVCGFIHLVIEGWFVLYYEDLLGDQAFLSQLWKEYAKGDSRYILGDNFTVCMETITACLWGPLSLWVVIAFLRQHPLRFILQLVVSVGQIYGDVLYFLTEHRDGFQHGELGHPLYFWFYFVFMNALWLVLPGVLVLDAVKHLTHAQSTLDAKATKAKSKKN",
# TMED2_HUMAN
"MVTLAELLVLLAALLATVSGYFVSIDAHAEECFFERVTSGTKMGLIFEVAEGGFLDIDVEITGPDNKGIYKGDRESSGKYTFAAHMDGTYKFCFSNRMSTMTPKIVMFTIDIGEAPKGQDMETEAHQNKLEEMINELAVAMTAVKHEQEYMEVRERIHRAINDNTNSRVVLWSFFEALVLVAMTLGQIYYLKRFFEVRRVV",
# DHC24_HUMAN
"MEPAVSLAVCALLFLLWVRLKGLEFVLIHQRWVFVCLFLLPLSLIFDIYYYVRAWVVFKLSSAPRLHEQRVRDIQKQVREWKEQGSKTFMCTGRPGWLTVSLRVGKYKKTHKNIMINLMDILEVDTKKQIVRVEPLVTMGQVTALLTSIGWTLPVLPELDDLTVGGLIMGTGIESSSHKYGLFQHICTAYELVLADGSFVRCTPSENSDLFYAVPWSCGTLGFLVAAEIRIIPAKKYVKLRFEPVRGLEAICAKFTHESQRQENHFVEGLLYSLDEAVIMTGVMTDEAEPSKLNSIGNYYKPWFFKHVENYLKTNREGLEYIPLRHYYHRHTRSIFWELQDIIPFGNNPIFRYLFGWMVPPKISLLKLTQGETLRKLYEQHHVVQDMLVPMKCLQQALHTFQNDIHVYPIWLCPFILPSQPGLVHPKGNEAELYIDIGAYGEPRVKHFEARSCMRQLEKFVRSVHGFQMLYADCYMNREEFWEMFDGSLYHKLREKLGCQDAFPEVYDKICKAARH",
# SC23B_HUMAN
"MATYLEFIQQNEERDGVRFSWNVWPSSRLEATRMVVPLACLLTPLKERPDLPPVQYEPVLCSRPTCKAVLNPLCQVDYRAKLWACNFCFQRNQFPPAYGGISEVNQPAELMPQFSTIEYVIQRGAQSPLIFLYVVDTCLEEDDLQALKESLQMSLSLLPPDALVGLITFGRMVQVHELSCEGISKSYVFRGTKDLTAKQIQDMLGLTKPAMPMQQARPAQPQEHPFASSRFLQPVHKIDMNLTDLLGELQRDPWPVTQGKRPLRSTGVALSIAVGLLEGTFPNTGARIMLFTGGPPTQGPGMVVGDELKIPIRSWHDIEKDNARFMKKATKHYEMLANRTAANGHCIDIYACALDQTGLLEMKCCANLTGGYMVMGDSFNTSLFKQTFQRIFTKDFNGDFRMAFGATLDVKTSRELKIAGAIGPCVSLNVKGPCVSENELGVGGTSQWKICGLDPTSTLGIYFEVVNQHNTPIPQGGRGAIQFVTHYQHSSTQRRIRVTTIARNWADVQSQLRHIEAAFDQEAAAVLMARLGVFRAESEEGPDVLRWLDRQLIRLCQKFGQYNKEDPTSFRLSDSFSLYPQFMFHLRRSPFLQVFNNSPDESSYYRHHFARQDLTQSLIMIQPILYSYSFHGPPEPVLLDSSSILADRILLMDTFFQIVIYLGETIAQWRKAGYQDMPEYENFKHLLQAPLDDAQEILQARFPMPRYINTEHGGSQARFLLSKVNPSQTHNNLYAWGQETGAPILTDDVSLQVFMDHLKKLAVSSAC",
# TRAM1_HUMAN
"MAIRKKSTKSPPVLSHEFVLQNHADIVSCVAMVFLLGLMFEITAKASIIFVTLQYNVTLPATEEQATESVSLYYYGIKDLATVFFYMLVAIIIHAVIQEYMLDKINRRMHFSKTKHSKFNESGQLSAFYLFACVWGTFILISENYISDPTILWRAYPHNLMTFQMKFFYISQLAYWLHAFPELYFQKTKKEDIPRQLVYIGLYLFHIAGAYLLNLNHLGLVLLVLHYFVEFLFHISRLFYFSNEKYQKGFSLWAVLFVLGRLLTLILSVLTVGFGLARAENQKLDFSTGNFNVLAVRIAVLASICVTQAFMMWKFINFQLRRWREHSAFQAPAVKKKPTVTKGRSSKKGTENGVNGTLTSNVADSPRNKKEKSS",
# NSDHL_HUMAN
"MEPAVSEPMRDQVARTHLTEDTPKVNADIEKVNQNQAKRCTVIGGSGFLGQHMVEQLLARGYAVNVFDIQQGFDNPQVRFFLGDLCSRQDLYPALKGVNTVFHCASPPPSSNNKELFYRVNYIGTKNVIETCKEAGVQKLILTSSASVIFEGVDIKNGTEDLPYAMKPIDYYTETKILQERAVLGANDPEKNFLTTAIRPHGIFGPRDPQLVPILIEAARNGKMKFVIGNGKNLVDFTFVENVVHGHILAAEQLSRDSTLGGKAFHITNDEPIPFWTFLSRILTGLNYEAPKYHIPYWVAYYLALLLSLLVMVISPVIQLQPTFTPMRVALAGTFHYYSCERAKKAMGYQPLVTMDDAMERTVQSFRHLRRVK",
# MSMO1_HUMAN
"MATNESVSIFSSASLAVEYVDSLLPENPLQEPFKNAWNYMLNNYTKFQIATWGSLIVHEALYFLFCLPGFLFQFIPYMKKYKIQKDKPETWENQWKCFKVLLFNHFCIQLPLICGTYYFTEYFNIPYDWERMPRWYFLLARCFGCAVIEDTWHYFLHRLLHHKRIYKYIHKVHHEFQAPFGMEAEYAHPLETLILGTGFFIGIVLLCDHVILLWAWVTIRLLETIDVHSGYDIPLNPLNLIPFYAGSRHHDFHHMNFIGNYASTFTWWDRIFGTDSQYNAYNEKRKKFEKKTE",
# EXT1_HUMAN
"MQAKKRYFILLSAGSCLALLFYFGGLQFRASRSHSRREEHSGRNGLHHPSPDHFWPRFPDALRPFVPWDQLENEDSSVHISPRQKRDANSSIYKGKKCRMESCFDFTLCKKNGFKVYVYPQQKGEKIAESYQNILAAIEGSRFYTSDPSQACLFVLSLDTLDRDQLSPQYVHNLRSKVQSLHLWNNGRNHLIFNLYSGTWPDYTEDVGFDIGQAMLAKASISTENFRPNFDVSIPLFSKDHPRTGGERGFLKFNTIPPLRKYMLVFKGKRYLTGIGSDTRNALYHVHNGEDVVLLTTCKHGKDWQKHKDSRCDRDNTEYEKYDYREMLHNATFCLVPRGRRLGSFRFLEALQAACVPVMLSNGWELPFSEVINWNQAAVIGDERLLLQIPSTIRSIHQDKILALRQQTQFLWEAYFSSVEKIVLTTLEIIQDRIFKHISRNSLIWNKHPGGLFVLPQYSSYLGDFPYYYANLGLKPPSKFTAVIHAVTPLVSQSQPVLKLLVAAAKSQYCAQIIVLWNCDKPLPAKHRWPATAVPVVVIEGESKVMSSRFLPYDNIITDAVLSLDEDTVLSTTEVDFAFTVWQSFPERIVGYPARSHFWDNSKERWGYTSKWTNDYSMVLTGAAIYHKYYHYLYSHYLPASLKNMVDQLANCEDILMNFLVSAVTKLPPIKVTQKKQYKETMMGQTSRASRWADPDHFAQRQSCMNTFASWFGYMPLIHSQMRLDPVLFKDQVSILRKKYRDIERL",
# RPE65_HUMAN
"MSIQVEHPAGGYKKLFETVEELSSPLTAHVTGRIPLWLTGSLLRCGPGLFEVGSEPFYHLFDGQALLHKFDFKEGHVTYHRRFIRTDAYVRAMTEKRIVITEFGTCAFPDPCKNIFSRFFSYFRGVEVTDNALVNVYPVGEDYYACTETNFITKINPETLETIKQVDLCNYVSVNGATAHPHIENDGTVYNIGNCFGKNFSIAYNIVKIPPLQADKEDPISKSEIVVQFPCSDRFKPSYVHSFGLTPNYIVFVETPVKINLFKFLSSWSLWGANYMDCFESNETMGVWLHIADKKRKKYLNNKYRTSPFNLFHHINTYEDNGFLIVDLCCWKGFEFVYNYLYLANLRENWEEVKKNARKAPQPEVRRYVLPLNIDKADTGKNLVTLPNTTATAILCSDETIWLEPEVLFSGPRQAFEFPQINYQKYCGKPYTYAYGLGLNHFVPDRLCKLNVKTKETWVWQEPDSYPSEPIFVSHPDALEEDDGVVLSVVVSPGAGQKPAYLLILNAKDLSEVARAEVEINIPVTFHGLFKKS",
# PTGIS_HUMAN
"MAWAALLGLLAALLLLLLLSRRRTRRPGEPPLDLGSIPWLGYALDFGKDAASFLTRMKEKHGDIFTILVGGRYVTVLLDPHSYDAVVWEPRTRLDFHAYAIFLMERIFDVQLPHYSPSDEKARMKLTLLHRELQALTEAMYTNLHAVLLGDATEAGSGWHEMGLLDFSYSFLLRAGYLTLYGIEALPRTHESQAQDRVHSADVFHTFRQLDRLLPKLARGSLSVGDKDHMCSVKSRLWKLLSPARLARRAHRSKWLESYLLHLEEMGVSEEMQARALVLQLWATQGNMGPAAFWLLLFLLKNPEALAAVRGELESILWQAEQPVSQTTTLPQKVLDSTPVLDSVLSESLRLTAAPFITREVVVDLAMPMADGREFNLRRGDRLLLFPFLSPQRDPEIYTDPEVFKYNRFLNPDGSEKKDFYKDGKRLKNYNMPWGAGHNHCLGRSYAVNSIKQFVFLVLVHLDLELINADVEIPEFDLSRYGFGLMQPEHDVPVRYRIRP",
# RTN1_HUMAN
"MAAPGDPQDELLPLAGPGSQWLRHRGEGENEAVTPKGATPAPQAGEPSPGLGARAREAASREAGSGPARQSPVAMETASTGVAGVSSAMDHTFSTTSKDGEGSCYTSLISDICYPPQEDSTYFTGILQKENGHVTISESPEELGTPGPSLPDVPGIESRGLFSSDSGIEMTPAESTEVNKILADPLDQMKAEAYKYIDITRPEEVKHQEQHHPELEDKDLDFKNKDTDISIKPEGVREPDKPAPVEGKIIKDHLLEESTFAPYIDDLSEEQRRAPQITTPVKITLTEIEPSVETTTQEKTPEKQDICLKPSPDTVPTVTVSEPEDDSPGSITPPSSGTEPSAAESQGKGSISEDELITAIKEAKGLSYETAENPRPVGQLADRPEVKARSGPPTIPSPLDHEASSAESGDSEIELVSEDPMAAEDALPSGYVSFGHVGGPPPSPASPSIQYSILREEREAELDSELIIESCDASSASEESPKREQDSPPMKPSALDAIREETGVRAEERAPSRRGLAEPGSFLDYPSTEPQPGPELPPGDGALEPETPMLPRKPEEDSSSNQSPAATKGPGPLGPGAPPPLLFLNKQKAIDLLYWRDIKQTGIVFGSFLLLLFSLTQFSVVSVVAYLALAALSATISFRIYKSVLQAVQKTDEGHPFKAYLELEITLSQEQIQKYTDCLQFYVNSTLKELRRLFLVQDLVDSLKFAVLMWLLTYVGALFNGLTLLLMAVVSMFTLPVVYVKHQAQIDQYLGLVRTHINAVVAKIQAKIPGAKRHAE",
# CP51A_HUMAN
"MAAAAGMLLLGLLQAGGSVLGQAMEKVTGGNLLSMLLIACAFTLSLVYLIRLAAGHLVQLPAGVKSPPYIFSPIPFLGHAIAFGKSPIEFLENAYEKYGPVFSFTMVGKTFTYLLGSDAAALLFNSKNEDLNAEDVYSRLTTPVFGKGVAYDVPNPVFLEQKKMLKSGLNIAHFKQHVSIIEKETKEYFESWGESGEKNVFEALSELIILTASHCLHGKEIRSQLNEKVAQLYADLDGGFSHAAWLLPGWLPLPSFRRRDRAHREIKDIFYKAIQKRRQSQEKIDDILQTLLDATYKDGRPLTDDEVAGMLIGLLLAGQHTSSTTSAWMGFFLARDKTLQKKCYLEQKTVCGENLPPLTYDQLKDLNLLDRCIKETLRLRPPIMIMMRMARTPQTVAGYTIPPGHQVCVSPTVNQRLKDSWVERLDFNPDRYLQDNPASGEKFAYVPFGAGRHRCIGENFAYVQIKTIWSTMLRLYEFDLIDGYFPTVNYTTMIHTPENPVIRYKRRSK",
# LTC4S_HUMAN
"MKDEVALLAAVTLLGVLLQAYFSLQVISARRAFRVSPPLTTGPPEFERVYRAQVNCSEYFPLFLATLWVAGIFFHEGAAALCGLVYLFARLRYFQGYARSAQLRLAPLYASARALWLLVALAALGLLAHFLPAALRAALLGRLRTLLPWA",
# T132A_HUMAN
"MCARMAGRTTAAPRGPYGPWLCLLVALALDVVRVDCGQAPLDPVYLPAALELLDAPEHFRVQQVGHYPPANSSLSSRSETFLLLQPWPRAQPLLRASYPPFATQQVVPPRVTEPHQRPVPWDVRAVSVEAAVTPAEPYARVLFHLKGQDWPPGSGSLPCARLHATHPAGTAHQACRFQPSLGACVVELELPSHWFSQASTTRAELAYTLEPAAEGPGGCGSGEENDPGEQALPVGGVELRPADPPQYQEVPLDEAVTLRVPDMPVRPGQLFSATLLLRHNFTASLLTLRIKVKKGLHVTAARPAQPTLWTAKLDRFKGSRHHTTLITCHRAGLTEPDSSPLELSEFLWVDFVVENSTGGGVAVTRPVTWQLEYPGQAPEAEKDKMVWEILVSERDIRALIPLAKAEELVNTAPLTGVPQHVPVRLVTVDGGGALVEVTEHVGCESANTQVLQVSEACDAVFVAGKESRGARGVRVDFWWRRLRASLRLTVWAPLLPLRIELTDTTLEQVRGWRVPGPAEGPAEPAAEASDEAERRARGCHLQYQRAGVRFLAPFAAHPLDGGRRLTHLLGPDWLLDVSHLVAPHARVLDSRVASLEGGRVVVGREPGVTSIEVRSPLSDSILGEQALAVTDDKVSVLELRVQPVMGISLTLSRGTAHPGEVTATCWAQSALPAPKQEVALSLWLSFSDHTVAPAELYDRRDLGLSVSAEEPGAILPAEEQGAQLGVVVSGAGAEGLPLHVALHPPEPCRRGRHRVPLASGTAWLGLPPASTPAPALPSSPAWSPPATEATMGGKRQVAGSVGGNTGVRGKFERAEEEARKEETEAREEEEEEEEEMVPAPQHVTELELGMYALLGVFCVAIFIFLVNGVVFVLRYQRKEPPDSATDPTSPQPHNWVWLGTDQEELSRQLDRQSPGPPKGEGSCPCESGGGGEAPTLAPGPPGGTTSSSSTLARKEAGGRRKRVEFVTFAPAPPAQSPEEPVGAPAVQSILVAGEEDIRWVCEDMGLKDPEELRNYMERIRGSS",
# MOGT2_HUMAN
"MVEFAPLFMPWERRLQTLAVLQFVFSFLALAEICTVGFIALLFTRFWLLTVLYAAWWYLDRDKPRQGGRHIQAIRCWTIWKYMKDYFPISLVKTAELDPSRNYIAGFHPHGVLAVGAFANLCTESTGFSSIFPGIRPHLMMLTLWFRAPFFRDYIMSAGLVTSEKESAAHILNRKGGGNLLGIIVGGAQEALDARPGSFTLLLRNRKGFVRLALTHGAPLVPIFSFGENDLFDQIPNSSGSWLRYIQNRLQKIMGISLPLFHGRGVFQYSFGLIPYRRPITTVVGKPIEVQKTLHPSEEEVNQLHQRYIKELCNLFEAHKLKFNIPADQHLEFC",
# GPAT3_HUMAN
"MEGAELAGKILSTWLTLVLGFILLPSVFGVSLGISEIYMKILVKTLEWATIRIEKGTPKESILKNSASVGIIQRDESPMEKGLSGLRGRDFELSDVFYFSKKGLEAIVEDEVTQRFSSEELVSWNLLTRTNVNFQYISLRLTMVWVLGVIVRYCVLLPLRVTLAFIGISLLVIGTTLVGQLPDSSLKNWLSELVHLTCCRICVRALSGTIHYHNKQYRPQKGGICVANHTSPIDVLILTTDGCYAMVGQVHGGLMGIIQRAMVKACPHVWFERSEMKDRHLVTKRLKEHIADKKKLPILIFPEGTCINNTSVMMFKKGSFEIGGTIHPVAIKYNPQFGDAFWNSSKYNMVSYLLRMMTSWAIVCDVWYMPPMTREEGEDAVQFANRVKSAIAIQGGLTELPWDGGLKRAKVKDIFKEEQQKNYSKMIVGNGSLS",
# ORML2_HUMAN
"MNVGVAHSEVNPNTRVMNSRGIWLAYIILVGLLHMVLLSIPFFSIPVVWTLTNVIHNLATYVFLHTVKGTPFETPDQGKARLLTHWEQMDYGLQFTSSRKFLSISPIVLYLLASFYTKYDAAHFLINTASLLSVLLPKLPQFHGVRVFGINKY",
# MEST_HUMAN
"MVRRDRLRRMREWWVQVGLLAVPLLAAYLHIPPPQLSPALHSWKSSGKFFTYKGLRIFYQDSVGVVGSPEIVVLLHGFPTSSYDWYKIWEGLTLRFHRVIALDFLGFGFSDKPRPHHYSIFEQASIVEALLRHLGLQNRRINLLSHDYGDIVAQELLYRYKQNRSGRLTIKSLCLSNGGIFPETHRPLLLQKLLKDGGVLSPILTRLMNFFVFSRGLTPVFGPYTRPSESELWDMWAGIRNNDGNLVIDSLLQYINQRKKFRRRWVGALASVTIPIHFIYGPLDPVNPYPEFLELYRKTLPRSTVSILDDHISHYPQLEDPMGFLNAYMGFINSF",
# PIGG_HUMAN
"MRLGSGTFATCCVAIEVLGIAVFLRGFFPAPVRSSARAEHGAEPPAPEPSAGASSNWTTLPPPLFSKVVIVLIDALRDDFVFGSKGVKFMPYTTYLVEKGASHSFVAEAKPPTVTMPRIKALMTGSLPGFVDVIRNLNSPALLEDSVIRQAKAAGKRIVFYGDETWVKLFPKHFVEYDGTTSFFVSDYTEVDNNVTRHLDKVLKRGDWDILILHYLGLDHIGHISGPNSPLIGQKLSEMDSVLMKIHTSLQSKERETPLPNLLVLCGDHGMSETGSHGASSTEEVNTPLILISSAFERKPGDIRHPKHVQQTDVAATLAIALGLPIPKDSVGSLLFPVVEGRPMREQLRFLHLNTVQLSKLLQENVPSYEKDPGFEQFKMSERLHGNWIRLYLEEKHSEVLFNLGSKVLRQYLDALKTLSLSLSAQVAQYDIYSMMVGTVVVLEVLTLLLLSVPQALRRKAELEVPLSSPGFSLLFYLVILVLSAVHVIVCTSAESSCYFCGLSWLAAGGVMVLASALLCVIVSVLTNVLVGGNTPRKNPMHPSSRWSELDLLILLGTAGHVLSLGASSFVEEEHQTWYFLVNTLCLALSQETYRNYFLGDDGEPPCGLCVEQGHDGATAAWQDGPGCDVLERDKGHGSPSTSEVLRGREKWMVLASPWLILACCRLLRSLNQTGVQWAHRPDLGHWLTSSDHKAELSVLAALSLLVVFVLVQRGCSPVSKAALALGLLGVYCYRAAIGSVRFPWRPDSKDISKGIIEARFVYVFVLGILFTGTKDLLKSQVIAADFKLKTVGLWEIYSGLVLLAALLFRPHNLPVLAFSLLIQTLMTKFIWKPLRHDAAEITVMHYWFGQAFFYFQGNSNNIATVDISAGFVGLDTYVEIPAVLLTAFGTYAGPVLWASHLVHFLSSETRSGSALSHACFCYALICSIPVFTYIVLVTSLRYHLFIWSVFSPKLLYEGMHLLITAAVCVFFTAMDQTRLTQS",
# S35D3_HUMAN
"MRQLCRGRVLGISVAIAHGVFSGSLNILLKFLISRYQFSFLTLVQCLTSSTAALSLELLRRLGLIAVPPFGLSLARSFAGVAVLSTLQSSLTLWSLRGLSLPMYVVFKRCLPLVTMLIGVLVLKNGAPSPGVLAAVLITTCGAALAGAGDLTGDPIGYVTGVLAVLVHAAYLVLIQKASADTEHGPLTAQYVIAVSATPLLVICSFASTDSIHAWTFPGWKDPAMVCIFVACILIGCAMNFTTLHCTYINSAVTTSFVGVVKSIATITVGMVAFSDVEPTSLFIAGVVVNTLGSIIYCVAKFMETRKQSNYEDLEAQPRGEEAQLSGDQLPFVMEELPGEGGNGRSEGGEAAGGPAQESRQEVRGSPRGVPLVAGSSEEGSRRSLKDAYLEVWRLVRGTRYMKKDYLIENEELPSP",
# LRRK2_HUMAN
"MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFTYSERASKLFQGKNIHVPLLIVLDSYMRVASVQQVGWSLLCKLIEVCPGTMQSLMGPQDVGNDWEVLGVHQLILKMLTVHNASVNLSVIGLKTLDLLLTSGKITLLILDEESDIFMLIFDAMHSFPANDEVQKLGCKALHVLFERVSEEQLTEFVENKDYMILLSALTNFKDEEEIVLHVLHCLHSLAIPCNNVEVLMSGNVRCYNIVVEAMKAFPMSERIQEVSCCLLHRLTLGNFFNILVLNEVHEFVVKAVQQYPENAALQISALSCLALLTETIFLNQDLEEKNENQENDDEGEEDKLFWLEACYKALTWHRKNKHVQEAACWALNNLLMYQNSLHEKIGDEDGHFPAHREVMLSMLMHSSSKEVFQASANALSTLLEQNVNFRKILLSKGIHLNVLELMQKHIHSPEVAESGCKMLNHLFEGSNTSLDIMAAVVPKILTVMKRHETSLPVQLEALRAILHFIVPGMPEESREDTEFHHKLNMVKKQCFKNDIHKLVLAALNRFIGNPGIQKCGLKVISSIVHFPDALEMLSLEGAMDSVLHTLQMYPDDQEIQCLGLSLIGYLITKKNVFIGTGHLLAKILVSSLYRFKDVAEIQTKGFQTILAILKLSASFSKLLVHHSFDLVIFHQMSSNIMEQKDQQFLNLCCKCFAKVAMDDYLKNVMLERACDQNNSIMVECLLLLGADANQAKEGSSLICQVCEKESSPKLVELLLNSGSREQDVRKALTISIGKGDSQIISLLLRRLALDVANNSICLGGFCIGKVEPSWLGPLFPDKTSNLRKQTNIASTLARMVIRYQMKSAVEEGTASGSDGNFSEDVLSKFDEWTFIPDSSMDSVFAQSDDLDSEGSEGSFLVKKKSNSISVGEFYRDAVLQRCSPNLQRHSNSLGPIFDHEDLLKRKRKILSSDDSLRSSKLQSHMRHSDSISSLASEREYITSLDLSANELRDIDALSQKCCISVHLEHLEKLELHQNALTSFPQQLCETLKSLTHLDLHSNKFTSFPSYLLKMSCIANLDVSRNDIGPSVVLDPTVKCPTLKQFNLSYNQLSFVPENLTDVVEKLEQLILEGNKISGICSPLRLKELKILNLSKNHISSLSENFLEACPKVESFSARMNFLAAMPFLPPSMTILKLSQNKFSCIPEAILNLPHLRSLDMSSNDIQYLPGPAHWKSLNLRELLFSHNQISILDLSEKAYLWSRVEKLHLSHNKLKEIPPEIGCLENLTSLDVSYNLELRSFPNEMGKLSKIWDLPLDELHLNFDFKHIGCKAKDIIRFLQQRLKKAVPYNRMKLMIVGNTGSGKTTLLQQLMKTKKSDLGMQSATVGIDVKDWPIQIRDKRKRDLVLNVWDFAGREEFYSTHPHFMTQRALYLAVYDLSKGQAEVDAMKPWLFNIKARASSSPVILVGTHLDVSDEKQRKACMSKITKELLNKRGFPAIRDYHFVNATEESDALAKLRKTIINESLNFKIRDQLVVGQLIPDCYVELEKIILSERKNVPIEFPVIDRKRLLQLVRENQLQLDENELPHAVHFLNESGVLLHFQDPALQLSDLYFVEPKWLCKIMAQILTVKVEGCPKHPKGIISRRDVEKFLSKKRKFPKNYMSQYFKLLEKFQIALPIGEEYLLVPSSLSDHRPVIELPHCENSEIIIRLYEMPYFPMGFWSRLINRLLEISPYMLSGRERALRPNRMYWRQGIYLNWSPEAYCLVGSEVLDNHPESFLKITVPSCRKGCILLGQVVDHIDSLMEEWFPGLLEIDICGEGETLLKKWALYSFNDGEEHQKILLDDLMKKAEEGDLLVNPDQPRLTIPISQIAPDLILADLPRNIMLNNDELEFEQAPEFLLGDGSFGSVYRAAYEGEEVAVKIFNKHTSLRLLRQELVVLCHLHHPSLISLLAAGIRPRMLVMELASKGSLDRLLQQDKASLTRTLQHRIALHVADGLRYLHSAMIIYRDLKPHNVLLFTLYPNAAIIAKIADYGIAQYCCRMGIKTSEGTPGFRAPEVARGNVIYNQQADVYSFGLLLYDILTTGGRIVEGLKFPNEFDELEIQGKLPDPVKEYGCAPWPMVEKLIKQCLKENPQERPTSAQVFDILNSAELVCLTRRILLPKNVIVECMVATHHNSRNASIWLGCGHTDRGQLSFLDLNTEGYTSEEVADSRILCLALVHLPVEKESWIVSGTQSGTLLVINTEDGKKRHTLEKMTDSVTCLYCNSFSKQSKQKNFLLVGTADGKLAIFEDKTVKLKGAAPLKILNIGNVSTPLMCLSESTNSTERNVMWGGCGTKIFSFSNDFTIQKLIETRTSQLFSYAAFSDSNIITVVVDTALYIAKQNSPVVEVWDKKTEKLCGLIDCVHFLREVMVKENKESKHKMSYSGRVKTLCLQKNTALWIGTGGGHILLLDLSTRRLIRVIYNFCNSVRVMMTAQLGSLKNVMLVLGYNRKNTEGTQKQKEIQSCLTVWDINLPHEVQNLEKHIEVRKELAEKMRRTSVE",
# ZFY27_HUMAN
"MQTSEREGSGPELSPSVMPEAPLESPPFPTKSPAFDLFNLVLSYKRLEIYLEPLKDAGDGVRYLLRWQMPLCSLLTCLGLNVLFLTLNEGAWYSVGALMISVPALLGYLQEVCRARLPDSELMRRKYHSVRQEDLQRGRLSRPEAVAEVKSFLIQLEAFLSRLCCTCEAAYRVLHWENPVVSSQFYGALLGTVCMLYLLPLCWVLTLLNSTLFLGNVEFFRVVSEYRASLQQRMNPKQEEHAFESPPPPDVGGKDGLMDSTPALTPTEDLTPGSVEEAEEAEPDEEFKDAIEETHLVVLEDDEGAPCPAEDELALQDNGFLSKNEVLRSKVSRLTERLRKRYPTNNFGNCTGCSATFSVLKKRRSCSNCGNSFCSRCCSFKVPKSSMGATAPEAQRETVFVCASCNQTLSK",
# WLS_HUMAN
"MAGAIIENMSTKKLCIVGGILLVFQIIAFLVGGLIAPGPTTAVSYMSVKCVDARKNHHKTKWFVPWGPNHCDKIRDIEEAIPREIEANDIVFSVHIPLPHMEMSPWFQFMLFILQLDIAFKLNNQIRENAEVSMDVSLAYRDDAFAEWTEMAHERVPRKLKCTFTSPKTPEHEGRYYECDVLPFMEIGSVAHKFYLLNIRLPVNEKKKINVGIGEIKDIRLVGIHQNGGFTKVWFAMKTFLTPSIFIIMVWYWRRITMMSRPPVLLEKVIFALGISMTFINIPVEWFSIGFDWTWMLLFGDIRQGIFYAMLLSFWIIFCGEHMMDQHERNHIAGYWKQVGPIAVGSFCLFIFDMCERGVQLTNPFYSIWTTDIGTELAMAFIIVAGICLCLYFLFLCFMVFQVFRNISGKQSSLPAMSKVRRLHYEGLIFRFKFLMLITLACAAMTVIFFIVSQVTEGHWKWGGVTVQVNSAFFTGIYGMWNLYVFALMFLYAPSHKNYGEDQSNGDLGVHSGEELQLTTTITHVDGPTEIYKLTRKEAQE",
# TSPO2_HUMAN
"MRLQGAIFVLLPHLGPILVWLFTRDHMSGWCEGPRMLSWCPFYKVLLLVQTAIYSVVGYASYLVWKDLGGGLGWPLALPLGLYAVQLTISWTVLVLFFTVHNPGLALLHLLLLYGLVVSTALIWHPINKLAALLLLPYLAWLTVTSALTYHLWRDSLCPVHQPQPTEKSD",
# CDKAL_HUMAN
"MPSASCDTLLDDIEDIVSQEDSKPQDRHFVRKDVVPKVRRRNTQKYLQEEENSPPSDSTIPGIQKIWIRTWGCSHNNSDGEYMAGQLAAYGYKITENASDADLWLLNSCTVKNPAEDHFRNSIKKAQEENKKIVLAGCVPQAQPRQDYLKGLSIIGVQQIDRVVEVVEETIKGHSVRLLGQKKDNGRRLGGARLDLPKIRKNPLIEIISINTGCLNACTYCKTKHARGNLASYPIDELVDRAKQSFQEGVCEIWLTSEDTGAYGRDIGTNLPTLLWKLVEVIPEGAMLRLGMTNPPYILEHLEEMAKILNHPRVYAFLHIPVQSASDSVLMEMKREYCVADFKRVVDFLKEKVPGITIATDIICGFPGETDQDFQETVKLVEEYKFPSLFINQFYPRPGTPAAKMEQVPAQVKKQRTKDLSRVFHSYSPYDHKIGERQQVLVTEESFDSKFYVAHNQFYEQVLVPKNPAFMGKMVEVDIYESGKHFMKGQPVSDAKVYTPSISKPLAKGEVSGLTKDFRNGLGNQLSSGSHTSAASQCDSASSRMVLPMPRLHQDCALRMSVGLALLGLLFAFFVKVYN",
# HACD4_HUMAN
"MGPLALPAWLQPRYRKNAYLFIYYLIQFCGHSWIFTNMTVRFFSFGKDSMVDTFYAIGLVMRLCQSVSLLELLHIYVGIESNHLLPRFLQLTERIIILFVVITSQEEVQEKYVVCVLFVFWNLLDMVRYTYSMLSVIGISYAVLTWLSQTLWMPIYPLCVLAEAFAIYQSLPYFESFGTYSTKLPFDLSIYFPYVLKIYLMMLFIGMYFTYSHLYSERRDILGIFPIKKKKM",
# LPCT4_HUMAN
"MSQGSPGDWAPLDPTPGPPASPNPFVHELHLSRLQRVKFCLLGALLAPIRVLLAFIVLFLLWPFAWLQVAGLSEEQLQEPITGWRKTVCHNGVLGLSRLLFFLLGFLRIRVRGQRASRLQAPVLVAAPHSTFFDPIVLLPCDLPKVVSRAENLSVPVIGALLRFNQAILVSRHDPASRRRVVEEVRRRATSGGKWPQVLFFPEGTCSNKKALLKFKPGAFIAGVPVQPVLIRYPNSLDTTSWAWRGPGVLKVLWLTASQPCSIVDVEFLPVYHPSPEESRDPTLYANNVQRVMAQALGIPATECEFVGSLPVIVVGRLKVALEPQLWELGKVLRKAGLSAGYVDAGAEPGRSRMISQEEFARQLQLSDPQTVAGAFGYFQQDTKGLVDFRDVALALAALDGGRSLEELTRLAFELFAEEQAEGPNRLLYKDGFSTILHLLLGSPHPAATALHAELCQAGSSQGLSLCQFQNFSLHDPLYGKLFSTYLRPPHTSRGTSQTPNASSPGNPTALANGTVQAPKQKGD",
# CR3L3_HUMAN
"MNTDLAAGKMASAACSMDPIDSFELLDLLFDRQDGILRHVELGEGWGHVKDQQVLPNPDSDDFLSSILGSGDSLPSSPLWSPEGSDSGISEDLPSDPQDTPPRSGPATSPAGCHPAQPGKGPCLSYHPGNSCSTTTPGPVIQVPEASVTIDLEMWSPGGRICAEKPADPVDLSPRCNLTVKDLLLSGSSGDLQQHHLGASYLLRPGAGHCQELVLTEDEKKLLAKEGITLPTQLPLTKYEERVLKKIRRKIRNKQSAQESRKKKKEYIDGLETRMSACTAQNQELQRKVLHLEKQNLSLLEQLKKLQAIVVQSTSKSAQTGTCVAVLLLSFALIILPSISPFGPNKTESPGDFAPVRVFSRTLHNDAASRVAADAVPGSEAPGPRPEADTTREESPGSPGADWGFQDTANLTNSTEELDNATLVLRNATEGLGQVALLDWVAPGPSTGSGRAGLEAAGDEL",
# SCAR3_HUMAN
"MKVRSAGGDGDALCVTEEDLAGDDEDMPTFPCTQKGRPGPRCSRCQKNLSLHTSVRILYLFLALLLVAVAVLASLVFRKVDSLSEDISLTQSIYDKKLVLMQKNLQGLDPKALNNCSFCHEAGQLGPEIRKLQEELEGIQKLLLAQEVQLDQTLQAQEVLSTTSRQISQEMGSCSFSIHQVNQSLGLFLAQVRGWQATTAGLDLSLKDLTQECYDVKAAVHQINFTVGQTSEWIHGIQRKTDEETLTLQKIVTDWQNYTRLFSGLRTTSTKTGEAVKNIQATLGASSQRISQNSESMHDLVLQVMGLQLQLDNISSFLDDHEENMHDLQYHTHYAQNRTVERFESLEGRMASHEIEIGTIFTNINATDNHVHSMLKYLDDVRLSCTLGFHTHAEELYYLNKSVSIMLGTTDLLRERFSLLSARLDLNVRNLSMIVEEMKAVDTQHGEILRNVTILRGAPGPPGPRGFKGDMGVKGPVGGRGPKGDPGSLGPLGPQGPQGQPGEAGPVGERGPVGPRGFPGLKGSKGSFGTGGPRGQPGPKGDIGPPGPEGPPGSPGPSGPQGKPGIAGKTGSPGQRGAMGPKGEPGIQGPPGLPGPPGPPGSQSFY",
# ATLA3_HUMAN
"MLSPQRVAAAASRGADDAMESSKPGPVQVVLVQKDQHSFELDEKALASILLQDHIRDLDVVVVSVAGAFRKGKSFILDFMLRYLYSQKESGHSNWLGDPEEPLTGFSWRGGSDPETTGIQIWSEVFTVEKPGGKKVAVVLMDTQGAFDSQSTVKDCATIFALSTMTSSVQIYNLSQNIQEDDLQQLQLFTEYGRLAMDEIFQKPFQTLMFLVRDWSFPYEYSYGLQGGMAFLDKRLQVKEHQHEEIQNVRNHIHSCFSDVTCFLLPHPGLQVATSPDFDGKLKDIAGEFKEQLQALIPYVLNPSKLMEKEINGSKVTCRGLLEYFKAYIKIYQGEDLPHPKSMLQATAEANNLAAAASAKDIYYNNMEEVCGGEKPYLSPDILEEKHCEFKQLALDHFKKTKKMGGKDFSFRYQQELEEEIKELYENFCKHNGSKNVFSTFRTPAVLFTGIVALYIASGLTGFIGLEVVAQLFNCMVGLLLIALLTWGYIRYSGQYRELGGAIDFGAAYVLEQASSHIGNSTQATVRDAVVGRPSMDKKAQ",
# MCTP1_HUMAN
"MEPRAAAAGEPEPPAASSSFQARLWKNLQLGVGRSKGGGGGRAGGPERRTADTPSPSPPPPVGTGNAPARGSGAGSRWSGFKKRKQVLDRVFSSSQPNLCCSSPEPLEPGGAGRAEQGSTLRRRIREHLLPAVKGPAAASGAAGGTPPGGRSPDSAPSSSSASSSLSSSPQPPPRGDRARDEGARRQGPGAHLCHQKSSSLPGTACLEQLLEPPPPPAEPARSPAESRAPETGEEHGSSQKIINTAGTSNAEVPLADPGMYQLDITLRRGQSLAARDRGGTSDPYVKFKIGGKEVFRSKIIHKNLNPVWEEKACILVDHLREPLYIKVFDYDFGLQDDFMGSAFLDLTQLELNRPTDVTLTLKDPHYPDHDLGIILLSVILTPKEGESRDVTMLMRKSWKRSSKELSENEVVGSYFSVKSLFWRTCGRPALPVLGFCRAELQNPYCKNVQFQTQSLRLSDLHRKSHLWRGIVSITLIEGRDLKAMDSNGLSDPYVKFRLGHQKYKSKIMPKTLNPQWREQFDFHLYEERGGVIDITAWDKDAGKRDDFIGRCQVDLSALSREQTHKLELQLEEGEGHLVLLVTLTASATVSISDLSVNSLEDQKEREEILKRYSPLRIFHNLKDVGFLQVKVIRAEGLMAADVTGKSDPFCVVELNNDRLLTHTVYKNLNPEWNKVFTFNIKDIHSVLEVTVYDEDRDRSADFLGKVAIPLLSIQNGEQKAYVLKNKQLTGPTKGVIYLEIDVIFNAVKASLRTLIPKEQKYIEEENRLSKQLLLRNFIRMKRCVMVLVNAAYYVNSCFDWDSPPRSLAAFVLFLFVVWNFELYMIPLVLLLLLTWNYFLIISGKDNRQRDTVVEDMLEDEEEEDDKDDKDSEKKGFINKIYAIQEVCVSVQNILDEVASFGERIKNTFNWTVPFLSWLAIVALCVFTAILYCIPLRYIVLVWGINKFTKKLRSPYAIDNNELLDFLSRVPSDVQVVQYQELKPDPSHSPYKRKKNNLG",
# RINT1_HUMAN
"MLPAGEIGASPAAPCCSESGDERKNLEEKSDINVTVLIGSKQVSEGTDNGDLPSYVSAFIEKEVGNDLKSLKKLDKLIEQRTVSKMQLEEQVLTISSEIPKRIRSALKNAEESKQFLNQFLEQETHLFSAINSHLLTAQPWMDDLGTMISQIEEIERHLAYLKWISQIEELSDNIQQYLMTNNVPEAASTLVSMAELDIKLQESSCTHLLGFMRATVKFWHKILKDKLTSDFEEILAQLHWPFIAPPQSQTVGLSRPASAPEIYSYLETLFCQLLKLQTSDELLTEPKQLPEKYSLPASPSVILPIQVMLTPLQKRFRYHFRGNRQTNVLSKPEWYLAQVLMWIGNHTEFLDEKIQPILDKVGSLVNARLEFSRGLMMLVLEKLATDIPCLLYDDNLFCHLVDEVLLFERELHSVHGYPGTFASCMHILSEETCFQRWLTVERKFALQKMDSMLSSEAAWVSQYKDITDVDEMKVPDCAETFMTLLLVITDRYKNLPTASRKLQFLELQKDLVDDFRIRLTQVMKEETRASLGFRYCAILNAVNYISTVLADWADNVFFLQLQQAALEVFAENNTLSKLQLGQLASMESSVFDDMINLLERLKHDMLTRQVDHVFREVKDAAKLYKKERWLSLPSQSEQAVMSLSSSACPLLLTLRDHLLQLEQQLCFSLFKIFWQMLVEKLDVYIYQEIILANHFNEGGAAQLQFDMTRNLFPLFSHYCKRPENYFKHIKEACIVLNLNVGSALLLKDVLQSASGQLPATAALNEVGIYKLAQQDVEILLNLRTNWPNTGK",
# TM214_HUMAN
"MATKTAGVGRWEVVKKGRRPGVGAGAGGRGGGRNRRALGEANGVWKYDLTPAIQTTSTLYERGFENIMKRQNKEQVPPPAVEPKKPGNKKQPKKVATPPNQNQKQGRFRSLEEALKALDVADLQKELDKSQSVFSGNPSIWLKDLASYLNYKLQAPLSEPTLSQHTHDYPYSLVSRELRGIIRGLLAKAAGSLELFFDHCLFTMLQELDKTPGESLHGYRICIQAILQDKPKIATANLGKFLELLRSHQSRPAKCLTIMWALGQAGFANLTEGLKVWLGIMLPVLGIKSLSPFAITYLDRLLLMHPNLTKGFGMIGPKDFFPLLDFAYMPNNSLTPSLQEQLCQLYPRLKVLAFGAKPDSTLHTYFPSFLSRATPSCPPEMKKELLSSLTECLTVDPLSASVWRQLYPKHLSQSSLLLEHLLSSWEQIPKKVQKSLQETIQSLKLTNQELLRKGSSNNQDVVTCDMACKGLLQQVQGPRLPWTRLLLLLLVFAVGFLCHDLRSHSSFQASLTGRLLRSSGFLPASQQACAKLYSYSLQGYSWLGETLPLWGSHLLTVVRPSLQLAWAHTNATVSFLSAHCASHLAWFGDSLTSLSQRLQIQLPDSVNQLLRYLRELPLLFHQNVLLPLWHLLLEALAWAQEHCHEACRGEVTWDCMKTQLSEAVHWTWLCLQDITVAFLDWALALISQQ",
# OFUT3_HUMAN
"MVRIQRRKLLASCLCVTATVFLLVTLQVMVELGKFERKEFKSSSLQDGHTKMEEAPTHLNSFLKKEGLTFNRKRKWELDSYPIMLWWSPLTGETGRLGQCGADACFFTINRTYLHHHMTKAFLFYGTDFNIDSLPLPRKAHHDWAVFHEESPKNNYKLFHKPVITLFNYTATFSRHSHLPLTTQYLESIEVLKSLRYLVPLQSKNKLRKRLAPLVYVQSDCDPPSDRDSYVRELMTYIEVDSYGECLRNKDLPQQLKNPASMDADGFYRIIAQYKFILAFENAVCDDYITEKFWRPLKLGVVPVYYGSPSITDWLPSNKSAILVSEFSHPRELASYIRRLDSDDRLYEAYVEWKLKGEISNQRLLTALRERKWGVQDVNQDNYIDAFECMVCTKVWANIRLQEKGLPPKRWEAEDTHLSCPEPTVFAFSPLRTPPLSSLREMWISSFEQSKKEAQALRWLVDRNQNFSSQEFWGLVFKD",
# TXD11_HUMAN
"MSECGGRGGGSSSSEDAEDEGGGGGGPAGSDCLSSSPTLATASSAGRLRRGLRGAFLMARQRPELLCGAVALGCALLLALKFTCSRAKDVIIPAKPPVSFFSLRSPVLDLFQGQLDYAEYVRRDSEVVLLFFYAPWCGQSIAARAEIEQAASRLSDQVLFVAINCWWNQGKCRKQKHFFYFPVIYLYHRSFGPIEYKGPMSAVYIEKFVRRVMKPLLYIPSQSELLDFLSNYEPGVLGYFEFSGSPQPPGYLTFFTSALHSLKKALESTSSPRALVSFTGEWHLETKIYVLDYLGTVRFGVITNKHLAKLVSLVHSGSVYLHRHFNTSLVFPREVLNYTAENICKWALENQETLFRWLRPHGGKSLLLNNELKKGPALFLFIPFNPLAESHPLIDEITEVALEYNNCHGDQVVERLLQHLRRVDAPVLESLALEVPAQLPDPPTITASPCCNTVVLPQWHSFSRTHNVCELCVNQTSGGMKPSSVSVPQCSFFEMAAALDSFYLKEQTFYHVASDSIECSNFLTSYSPFSYYTACCRTISRGVSGFIDSEQGVFEAPTVAFSSLEKKCEVDAPSSVPHIEENRYLFPEVDMTSTNFTGLSCRTNKTLNIYLLDSNLFWLYAERLGAPSSTQVKEFAAIVDVKEESHYILDPKQALMKLTLESFIQNFSVLYSPLKRHLIGSGSAQFPSQHLITEVTTDTFWEVVLQKQDVLLLYYAPWCGFCPSLNHIFIQLARNLPMDTFTVARIDVSQNDLPWEFMVDRLPTVLFFPCNRKDLSVKYPEDVPITLPNLLRFILHHSDPASSPQNVANSPTKECLQSEAVLQRGHISHLEREIQKLRAEISSLQRAQVQVESQLSSARRDEHRLRQQQRALEEQHSLLHAHSEQLQALYEQKTRELQELARKLQELADASENLLTENTWLKILVATMERKLEGRDGAESLAAQREVHPKQPEPSATPQLPGSSPPPANVSATLVSERNKENRTD",
# LCTL_HUMAN
"MKPVWVATLLWMLLLVPRLGAARKGSPEEASFYYGTFPLGFSWGVGSSAYQTEGAWDQDGKGPSIWDVFTHSGKGKVLGNETADVACDGYYKVQEDIILLRELHVNHYRFSLSWPRLLPTGIRAEQVNKKGIEFYSDLIDALLSSNITPIVTLHHWDLPQLLQVKYGGWQNVSMANYFRDYANLCFEAFGDRVKHWITFSDPRAMAEKGYETGHHAPGLKLRGTGLYKAAHHIIKAHAKAWHSYNTTWRSKQQGLVGISLNCDWGEPVDISNPKDLEAAERYLQFCLGWFANPIYAGDYPQVMKDYIGRKSAEQGLEMSRLPVFSLQEKSYIKGTSDFLGLGHFTTRYITERNYPSRQGPSYQNDRDLIELVDPNWPDLGSKWLYSVPWGFRRLLNFAQTQYGDPPIYVMENGASQKFHCTQLCDEWRIQYLKGYINEMLKAIKDGANIKGYTSWSLLDKFEWEKGYSDRYGFYYVEFNDRNKPRYPKASVQYYKKIIIANGFPNPREVESWYLKALETCSINNQMLAAEPLLSHMQMVTEIVVPTVCSLCVLITAVLLMLLLRRQS",
# LCLT1_HUMAN
"MHSRGREIVVLLNPWSINEAVSSYCTYFIKQDSKSFGIMVSWKGIYFILTLFWGSFFGSIFMLSPFLPLMFVNPSWYRWINNRLVATWLTLPVALLETMFGVKVIITGDAFVPGERSVIIMNHRTRMDWMFLWNCLMRYSYLRLEKICLKASLKGVPGFGWAMQAAAYIFIHRKWKDDKSHFEDMIDYFCDIHEPLQLLIFPEGTDLTENSKSRSNAFAEKNGLQKYEYVLHPRTTGFTFVVDRLREGKNLDAVHDITVAYPHNIPQSEKHLLQGDFPREIHFHVHRYPIDTLPTSKEDLQLWCHKRWEEKEERLRSFYQGEKNFYFTGQSVIPPCKSELRVLVVKLLSILYWTLFSPAMCLLIYLYSLVKWYFIITIVIFVLQERIFGGLEIIELACYRLLHKQPHLNSKKNE",
# HACD2_HUMAN
"MAAVAATAAAKGNGGGGGRAGAGDASGTRKKKGPGPLATAYLVIYNVVMTAGWLVIAVGLVRAYLAKGSYHSLYYSIEKPLKFFQTGALLEILHCAIGIVPSSVVLTSFQVMSRVFLIWAVTHSVKEVQSEDSVLLFVIAWTITEIIRYSFYTFSLLNHLPYLIKWARYTLFIVLYPMGVSGELLTIYAALPFVRQAGLYSISLPNKYNFSFDYYAFLILIMISYIPIFPQLYFHMIHQRRKILSHTEEHKKFE",
# CERS6_HUMAN
"MAGILAWFWNERFWLPHNVTWADLKNTEEATFPQAEDLYLAFPLAFCIFMVRLIFERFVAKPCAIALNIQANGPQIAPPNAILEKVFTAITKHPDEKRLEGLSKQLDWDVRSIQRWFRQRRNQEKPSTLTRFCESMWRFSFYLYVFTYGVRFLKKTPWLWNTRHCWYNYPYQPLTTDLHYYYILELSFYWSLMFSQFTDIKRKDFGIMFLHHLVSIFLITFSYVNNMARVGTLVLCLHDSADALLEAAKMANYAKFQKMCDLLFVMFAVVFITTRLGIFPLWVLNTTLFESWEIVGPYPSWWVFNLLLLLVQGLNCFWSYLIVKIACKAVSRGKVSKDDRSDIESSSDEEDSEPPGKNPHTATTTNGTSGTNGYLLTGSCSMDD",
# ALKMO_HUMAN
"MKNPEAQQDVSVSQGFRMLFYTMKPSETSFQTLEEVPDYVKKATPFFISLMLLELVVSWILKGKPPGRLDDALTSISAGVLSRLPSLFFRSIELTSYIYIWENYRLFNLPWDSPWTWYSAFLGVDFGYYWFHRMAHEVNIMWAGHQTHHSSEDYNLSTALRQSVLQIYTSWIFYSPLALFIPPSVYAVHLQFNLLYQFWIHTEVINNLGPLELILNTPSHHRVHHGRNRYCIDKNYAGVLIIWDKIFGTFEAENEKVVYGLTHPINTFEPIKVQFHHLFSIWTTFWATPGFFNKFSVIFKGPGWGPGKPRLGLSEEIPEVTGKEVPFSSSSSQLLKIYTVVQFALMLAFYEETFADTAALSQVTLLLRVCFIILTLTSIGFLLDQRPKAAIMETLRCLMFLMLYRFGHLKPLVPSLSSAFEIVFSICIAFWGVRSMKQLTSHPWK",
# PLPL7_HUMAN
"MEEEKDDSPQLTGIAVGALLALALVGVLILFMFRRLRQFRQAQPTPQYRFRKRDKVMFYGRKIMRKVTTLPNTLVENTALPRQRARKRTKVLSLAKRILRFKKEYPALQPKEPPPSLLEADLTEFDVKNSHLPSEVLYMLKNVRVLGHFEKPLFLELCKHIVFVQLQEGEHVFQPREPDPSICVVQDGRLEVCIQDTDGTEVVVKEVLAGDSVHSLLSILDIITGHAAPYKTVSVRAAIPSTILRLPAAAFHGVFEKYPETLVRVVQIIMVRLQRVTFLALHNYLGLTTELFNAESQAIPLVSVASVAAGKAKKQVFYGEEERLKKPPRLQESCDSDHGGGRPAAAGPLLKRSHSVPAPSIRKQILEELEKPGAGDPDPSAPQGGPGSATSDLGMACDRARVFLHSDEHPGSSVASKSRKSVMVAEIPSTVSQHSESHTDETLASRKSDAIFRAAKKDLLTLMKLEDSSLLDGRVALLHVPAGTVVSRQGDQDASILFVVSGLLHVYQRKIGSQEDTCLFLTRPGEMVGQLAVLTGEPLIFTVKANRDCSFLSISKAHFYEIMRKQPTVVLGVAHTVVKRMSSFVRQIDFALDWVEVEAGRAIYRQGDKSDCTYIMLSGRLRSVIRKDDGKKRLAGEYGRGDLVGVVETLTHQARATTVHAVRDSELAKLPAGALTSIKRRYPQVVTRLIHLLGEKILGSLQQGPVTGHQLGLPTEGSKWDLGNPAVNLSTVAVMPVSEEVPLTAFALELEHALSAIGPTLLLTSDNIKRRLGSAALDSVHEYRLSSWLGQQEDTHRIVLYQADGTLTPWTQRCVRQADCILIVGLGDQEPTVGELERMLESTAVRAQKQLILLHREEGPAPARTVEWLNMRSWCSGHLHLCCPRRVFSRRSLPKLVEMYKHVFQRPPDRHSDFSRLARVLTGNAIALVLGGGGARGCAQVGVLKALAECGIPVDMVGGTSIGAFVGALYSEERNYSQMRIRAKQWAEGMTSLMKAALDLTYPITSMFSGAGFNSSIFSVFKDQQIEDLWIPYFAITTDITASAMRVHTDGSLWWYVRASMSLSGYMPPLCDPKDGHLLMDGGYINNLPADVARSMGAKVVIAIDVGSRDETDLTNYGDALSGWWLLWKRWNPLATKVKVLNMAEIQTRLAYVCCVRQLEVVKSSDYCEYLRPPIDSYSTLDFGKFNEICEVGYQHGRTVFDIWGRSGVLEKMLRDQQGPSKKPASAVLTCPNASFTDLAEIVSRIEPAKPAMVDDESDYQTEYEEELLDVPRDAYADFQSTSAQQGSDLEDESSLRHRHPSLAFPKLSEGSSDQDG",
# CR3L2_HUMAN
"MEVLESGEQGVLQWDRKLSELSEPGDGEALMYHTHFSELLDEFSQNVLGQLLNDPFLSEKSVSMEVEPSPTSPAPLIQAEHSYSLCEEPRAQSPFTHITTSDSFNDDEVESEKWYLSTDFPSTSIKTEPVTDEPPPGLVPSVTLTITAISTPLEKEEPPLEMNTGVDSSCQTIIPKIKLEPHEVDQFLNFSPKEAPVDHLHLPPTPPSSHGSDSEGSLSPNPRLHPFSLPQTHSPSRAAPRAPSALSSSPLLTAPHKLQGSGPLVLTEEEKRTLIAEGYPIPTKLPLSKSEEKALKKIRRKIKNKISAQESRRKKKEYMDSLEKKVESCSTENLELRKKVEVLENTNRTLLQQLQKLQTLVMGKVSRTCKLAGTQTGTCLMVVVLCFAVAFGSFFQGYGPYPSATKMALPSQHSLQEPYTASVVRSRNLLIYEEHSPPEESSSPGSAGELGGWDRGSSLLRVSGLESRPDVDLPHFIISNETSLEKSVLLELQQHLVSAKLEGNETLKVVELDRRVNTTF",
# IKIP_HUMAN
"MSEVKSRKKSGPKGAPAAEPGKRSEGGKTPVARSSGGGGWADPRTCLSLLSLGTCLGLAWFVFQQSEKFAKVENQYQLLKLETNEFQQLQSKISLISEKWQKSEAIMEQLKSFQIIAHLKRLQEEINEVKTWSNRITEKQDILNNSLTTLSQDITKVDQSTTSMAKDVGLKITSVKTDIRRISGLVTDVISLTDSVQELENKIEKVEKNTVKNIGDLLSSSIDRTATLRKTASENSQRINSVKKTLTELKSDFDKHTDRFLSLEGDRAKVLKTVTFANDLKPKVYNLKKDFSRLEPLVNDLTLRIGRLVTDLLQREKEIAFLSEKISNLTIVQAEIKDIKDEIAHISDMN",
# TLC3B_HUMAN
"MLTPMVAGGVVFPGLFLLSKNTLQRLPQLRWEEADAVIVSARLVSSVQAIMASTAGYIVSTSCKHIIDDQHWLSSAYTQFAVPYFIYDIYAMFLCHWHKHQVKGHGGDDGAARAPGSTWAIARGYLHKEFLMVLHHAAMVLVCFPLSVVWRQGKGDFFLGCMLMAEVSTPFVCLGKILIQYKQQHTLLHKVNGALMLLSFLCCRVLLFPYLYWAYGRHAGLPLLAVPLAIPAHVNLGAALLLAPQLYWFFLICRGACRLFWPRSRPPPACQAQD",
# PGAP1_HUMAN
"MFLHSVNLWNLAFYVFMVFLATLGLWDVFFGFEENKCSMSYMFEYPEYQKIELPKKLAKRYPAYELYLYGEGSYAEEHKILPLTGIPVLFLPGNAGSYKQVRSIGSIALRKAEDIDFKYHFDFFSVNFNEELVALYGGSLQKQTKFVHECIKTILKLYKGQEFAPKSVAIIGHSMGGLVARALLTLKNFKHDLINLLITQATPHVAPVMPLDRFITDFYTTVNNYWILNARHINLTTLSVAGGFRDYQVRSGLTFLPKLSHHTSALSVVSSAVPKTWVSTDHLSIVWCKQLQLTTVRAFFDLIDADTKQITQNSKKKLSVLYHHFIRHPSKHFEENPAIISDLTGTSMWVLVKVSKWTYVAYNESEKIYFTFPLENHRKIYTHVYCQSTMLDTNSWIFACINSTSMCLQGVDLSWKAELLPTIKYLTLRLQDYPSLSHLVVYVPSVRGSKFVVDCEFFKKEKRYIQLPVTHLFSFGLSSRKVVLNTNGLYYNLELLNFGQIYQAFKINVVSKCSAVKEEITSIYRLHIPWSYEDSLTIAQAPSSTEISLKLHIAQPENNTHVALFKMYTSSDCRYEVTVKTSFSQILGQVVRFHGGALPAYVVSNILLAYRGQLYSLFSTGCCLEYATMLDKEAKPYKVDPFVIIIKFLLGYKWFKELWDVLLLPELDAVILTCQSMCFPLISLILFLFGTCTAYWSGLLSSASVRLLSSLWLALKRPSELPKDIKMISPDLPFLTIVLIIVSWTTCGALAILLSYLYYVFKVVHLQASLTTFKNSQPVNPKHSRRSEKKSNHHKDSSIHHLRLSANDAEDSLRMHSTVINLLTWIVLLSMPSLIYWLKNLRYYFKLNPDPCKPLAFILIPTMAILGNTYTVSIKSSKLLKTTSQFPLPLAVGVIAFGSAHLYRLPCFVFIPLLLHALCNFM",
# NB5R4_HUMAN
"MLNVPSQSFPAPRSQQRVASGGRSKVPLKQGRSLMDWIRLTKSGKDLTGLKGRLIEVTEEELKKHNKKDDCWICIRGFVYNVSPYMEYHPGGEDELMRAAGSDGTELFDQVHRWVNYESMLKECLVGRMAIKPAVLKDYREEEKKVLNGMLPKSQVTDTLAKEGPSYPSYDWFQTDSLVTIAIYTKQKDINLDSIIVDHQNDSFRAETIIKDCLYLIHIGLSHEVQEDFSVRVVESVGKIEIVLQKKENTSWDFLGHPLKNHNSLIPRKDTGLYYRKCQLISKEDVTHDTRLFCLMLPPSTHLQVPIGQHVYLKLPITGTEIVKPYTPVSGSLLSEFKEPVLPNNKYIYFLIKIYPTGLFTPELDRLQIGDFVSVSSPEGNFKISKFQELEDLFLLAAGTGFTPMVKILNYALTDIPSLRKVKLMFFNKTEDDIIWRSQLEKLAFKDKRLDVEFVLSAPISEWNGKQGHISPALLSEFLKRNLDKSKVLVCICGPVPFTEQGVRLLHDLNFSKNEIHSFTA",
# FA2H_HUMAN
"MAPAPPPAASFSPSEVQRRLAAGACWVRRGARLYDLSSFVRHHPGGEQLLRARAGQDISADLDGPPHRHSANARRWLEQYYVGELRGEQQGSMENEPVALEETQKTDPAMEPRFKVVDWDKDLVDWRKPLLWQVGHLGEKYDEWVHQPVTRPIRLFHSDLIEGLSKTVWYSVPIIWVPLVLYLSWSYYRTFAQGNVRLFTSFTTEYTVAVPKSMFPGLFMLGTFLWSLIEYLIHRFLFHMKPPSDSYYLIMLHFVMHGQHHKAPFDGSRLVFPPVPASLVIGVFYLCMQLILPEAVGGTVFAGGLLGYVLYDMTHYYLHFGSPHKGSYLYSLKAHHVKHHFAHQKSGFGISTKLWDYCFHTLTPEKPHLKTQ",
# PCAT2_HUMAN
"MSRCAQAAEVAATVPGAGVGNVGLRPPMVPRQASFFPPPVPNPFVQQTQIGSARRVQIVLLGIILLPIRVLLVALILLLAWPFAAISTVCCPEKLTHPITGWRRKITQTALKFLGRAMFFSMGFIVAVKGKIASPLEAPVFVAAPHSTFFDGIACVVAGLPSMVSRNENAQVPLIGRLLRAVQPVLVSRVDPDSRKNTINEIIKRTTSGGEWPQILVFPEGTCTNRSCLITFKPGAFIPGVPVQPVLLRYPNKLDTVTWTWQGYTFIQLCMLTFCQLFTKVEVEFMPVQVPNDEEKNDPVLFANKVRNLMAEALGIPVTDHTYEDCRLMISAGQLTLPMEAGLVEFTKISRKLKLDWDGVRKHLDEYASIASSSKGGRIGIEEFAKYLKLPVSDVLRQLFALFDRNHDGSIDFREYVIGLAVLCNPSNTEEIIQVAFKLFDVDEDGYITEEEFSTILQASLGVPDLDVSGLFKEIAQGDSISYEEFKSFALKHPEYAKIFTTYLDLQTCHVFSLPKEVQTTPSTASNKVSPEKHEESTSDKKDD",
# EPMIP_HUMAN
"MWMTPKRSKMEVDEALVFRPEWTQRYLVVEPPEGDGALCLVCRRLIVATRERDVRRHYEAEHEYYERYVADGERAALVERLRQGDLPVASFTPEERAARAGLGLCRLLALKGRGWGEGDFVYQCMEVLLREVLPEHVSVLQGVDLSPDITRQRILSIDRNLRNQLFNRARDFKAYSLALDDQAFVAYENYLLVFIRGVGPELEVQEDLLTIINLTHHFSVGALMSAILESLQTAGLSLQRMVGLTTTHTLRMIGENSGLVSYMREKAVSPNCWNVIHYSGFLHLELLSSYDVDVNQIINTISEWIVLIKTRGVRRPEFQTLLTESESEHGERVNGRCLNNWLRRGKTLKLIFSLRKEMEAFLVSVGATTVHFSDKQWLCDFGFLVDIMEHLRELSEELRVSKVFAAAAFDHICTFEVKLNLFQRHIEEKNLTDFPALREVVDELKQQNKEDEKIFDPDRYQMVICRLQKEFERHFKDLRFIKKDLELFSNPFNFKPEYAPISVRVELTKLQANTNLWNEYRIKDLGQFYAGLSAESYPIIKGVACKVASLFDSNQICEKAFSYLTRNQHTLSQPLTDEHLQALFRVATTEMEPGWDDLVRERNESNP",
# TMC6_HUMAN
"MAQPLAFILDVPETPGDQGQGPSPYDESEVHDSFQQLIQEQSQCTAQEGLELQQREREVTGSSQQTLWRPEGTQSTATLRILASMPSRTIGRSRGAIISQYYNRTVQLRCRSSRPLLGNFVRSAWPSLRLYDLELDPTALEEEEKQSLLVKELQSLAVAQRDHMLRGMPLSLAEKRSLREKSRTPRGKWRGQPGSGGVCSCCGRLRYACVLALHSLGLALLSALQALMPWRYALKRIGGQFGSSVLSYFLFLKTLLAFNALLLLLLVAFIMGPQVAFPPALPGPAPVCTGLELLTGAGCFTHTVMYYGHYSNATLNQPCGSPLDGSQCTPRVGGLPYNMPLAYLSTVGVSFFITCITLVYSMAHSFGESYRVGSTSGIHAITVFCSWDYKVTQKRASRLQQDNIRTRLKELLAEWQLRHSPRSVCGRLRQAAVLGLVWLLCLGTALGCAVAVHVFSEFMIQSPEAAGQEAVLLVLPLVVGLLNLGAPYLCRVLAALEPHDSPVLEVYVAICRNLILKLAILGTLCYHWLGRRVGVLQGQCWEDFVGQELYRFLVMDFVLMLLDTLFGELVWRIISEKKLKRRRKPEFDIARNVLELIYGQTLTWLGVLFSPLLPAVQIIKLLLVFYVKKTSLLANCQAPRRPWLASHMSTVFLTLLCFPAFLGAAVFLCYAVWQVKPSSTCGPFRTLDTMYEAGRVWVRHLEAAGPRVSWLPWVHRYLMENTFFVFLVSALLLAVIYLNIQVVRGQRKVICLLKEQISNEGEDKIFLINKLHSIYERKEREERSRVGTTEEAAAPPALLTDEQDA",
# SYVN1_HUMAN
"MFRTAVMMAASLALTGAVVAHAYYLKHQFYPTVVYLTKSSPSMAVLYIQAFVLVFLLGKVMGKVFFGQLRAAEMEHLLERSWYAVTETCLAFTVFRDDFSPRFVALFTLLLFLKCFHWLAEDRVDFMERSPNISWLFHCRIVSLMFLLGILDFLFVSHAYHSILTRGASVQLVFGFEYAILMTMVLTIFIKYVLHSVDLQSENPWDNKAVYMLYTELFTGFIKVLLYMAFMTIMIKVHTFPLFAIRPMYLAMRQFKKAVTDAIMSRRAIRNMNTLYPDATPEELQAMDNVCIICREEMVTGAKRLPCNHIFHTSCLRSWFQRQQTCPTCRMDVLRASLPAQSPPPPEPADQGPPPAPHPPPLLPQPPNFPQGLLPPFPPGMFPLWPPMGPFPPVPPPPSSGEAVAPPSTSAAALSRPSGAATTTAAGTSATAASATASGPGSGSAPEAGPAPGFPFPPPWMGMPLPPPFAFPPMPVPPAGFAGLTPEELRALEGHERQHLEARLQSLRNIHTLLDAAMLQINQYLTVLASLGPPRPATSVNSTEETATTVVAAASSTSIPSSEATTPTPGASPPAPEMERPPAPESVGTEEMPEDGEPDAAELRRRRLQKLESPVAH",
# LYRIC_HUMAN
"MAARSWQDELAQQAEEGSARLREMLSVGLGFLRTELGLDLGLEPKRYPGWVILVGTGALGLLLLFLLGYGWAAACAGARKKRRSPPRKREEAAAVPAAAPDDLALLKNLRSEEQKKKNRKKLSEKPKPNGRTVEVAEGEAVRTPQSVTAKQPPEIDKKNEKSKKNKKKSKSDAKAVQNSSRHDGKEVDEGAWETKISHREKRQQRKRDKVLTDSGSLDSTIPGIENTITVTTEQLTTASFPVGSKKNKGDSHLNVQVSNFKSGKGDSTLQVSSGLNENLTVNGGGWNEKSVKLSSQISAGEEKWNSVSPASAGKRKTEPSAWSQDTGDANTNGKDWGRSWSDRSIFSGIGSTAEPVSQSTTSDYQWDVSRNQPYIDDEWSGLNGLSSADPNSDWNAPAEEWGNWVDEERASLLKSQEPIPDDQKVSDDDKEKGEGALPTGKSKKKKKKKKKQGEDNSTAQDTEELEKEIREDLPVNTSKTRPKQEKAFSLKTISTSDPAEVLVKNSQPIKTLPPATSTEPSVILSKSDSDKSSSQVPPILQETDKSKSNTKQNSVPPSQTKSETSWESPKQIKKKKKARRET",
# GPAT4_HUMAN
"MFLLLPFDSLIVNLLGISLTVLFTLLLVFIIVPAIFGVSFGIRKLYMKSLLKIFAWATLRMERGAKEKNHQLYKPYTNGIIAKDPTSLEEEIKEIRRSGSSKALDNTPEFELSDIFYFCRKGMETIMDDEVTKRFSAEELESWNLLSRTNYNFQYISLRLTVLWGLGVLIRYCFLLPLRIALAFTGISLLVVGTTVVGYLPNGRFKEFMSKHVHLMCYRICVRALTAIITYHDRENRPRNGGICVANHTSPIDVIILASDGYYAMVGQVHGGLMGVIQRAMVKACPHVWFERSEVKDRHLVAKRLTEHVQDKSKLPILIFPEGTCINNTSVMMFKKGSFEIGATVYPVAIKYDPQFGDAFWNSSKYGMVTYLLRMMTSWAIVCSVWYLPPMTREADEDAVQFANRVKSAIARQGGLVDLLWDGGLKREKVKDTFKEEQQKLYSKMIVGNHKDRSRS",
# KTN1_HUMAN
"MEFYESAYFIVLIPSIVITVIFLFFWLFMKETLYDEVLAKQKREQKLIPTKTDKKKAEKKKNKKKEIQNGNLHESDSESVPRDFKLSDALAVEDDQVAPVPLNVVETSSSVRERKKKEKKQKPVLEEQVIKESDASKIPGKKVEPVPVTKQPTPPSEAAASKKKPGQKKSKNGSDDQDKKVETLMVPSKRQEALPLHQETKQESGSGKKKASSKKQKTENVFVDEPLIHATTYIPLMDNADSSPVVDKREVIDLLKPDQVEGIQKSGTKKLKTETDKENAEVKFKDFLLSLKTMMFSEDEALCVVDLLKEKSGVIQDALKKSSKGELTTLIHQLQEKDKLLAAVKEDAAATKDRCKQLTQEMMTEKERSNVVITRMKDRIGTLEKEHNVFQNKIHVSYQETQQMQMKFQQVREQMEAEIAHLKQENGILRDAVSNTTNQLESKQSAELNKLRQDYARLVNELTEKTGKLQQEEVQKKNAEQAATQLKVQLQEAERRWEEVQSYIRKRTAEHEAAQQDLQSKFVAKENEVQSLHSKLTDTLVSKQQLEQRLMQLMESEQKRVNKEESLQMQVQDILEQNEALKAQIQQFHSQIAAQTSASVLAEELHKVIAEKDKQIKQTEDSLASERDRLTSKEEELKDIQNMNFLLKAEVQKLQALANEQAAAAHELEKMQQSVYVKDDKIRLLEEQLQHEISNKMEEFKILNDQNKALKSEVQKLQTLVSEQPNKDVVEQMEKCIQEKDEKLKTVEELLETGLIQVATKEEELNAIRTENSSLTKEVQDLKAKQNDQVSFASLVEELKKVIHEKDGKIKSVEELLEAELLKVANKEKTVQDLKQEIKALKEEIGNVQLEKAQQLSITSKVQELQNLLKGKEEQMNTMKAVLEEKEKDLANTGKWLQDLQEENESLKAHVQEVAQHNLKEASSASQFEELEIVLKEKENELKRLEAMLKERESDLSSKTQLLQDVQDENKLFKSQIEQLKQQNYQQASSFPPHEELLKVISEREKEISGLWNELDSLKDAVEHQRKKNNDLREKNWEAMEALASTEKMLQDKVNKTSKERQQQVEAVELEAKEVLKKLFPKVSVPSNLSYGEWLHGFEKKAKECMAGTSGSEEVKVLEHKLKEADEMHTLLQLECEKYKSVLAETEGILQKLQRSVEQEENKWKVKVDESHKTIKQMQSSFTSSEQELERLRSENKDIENLRREREHLEMELEKAEMERSTYVTEVRELKDLLTELQKKLDDSYSEAVRQNEELNLLKAQLNETLTKLRTEQNERQKVAGDLHKAQQSLELIQSKIVKAAGDTTVIENSDVSPETESSEKETMSVSLNQTVTQLQQLLQAVNQQLTKEKEHYQVLE",
# PIGZ_HUMAN
"MQICGSSVASVAAGTSFQVLGPVCWQQLDLKMAVRVLWGGLSLLRVLWCLLPQTGYVHPDEFFQSPEVMAEDILGVQAARPWEFYPSSSCRSVLFPLLISGSTFWLLRLWEELGPWPGLVSGYALLVGPRLLLTALSFALDGAVYHLAPPMGADRWNALALLSGSYVTLVFYTRTFSNTIEGLLFTWLLVLVSSHVTWGPTRKEPAPGPRWRSWLLGGIVAAGFFNRPTFLAFAVVPLYLWGTRGATNPGLKSLTREALVLLPGAALTAAVFVATDSWYFSSPATSRNLVLTPVNFLHYNLNPQNLARHGTHARLTHLAVNGFLLFGVLHAQALQAAWQRLQVGLQASAQMGLLRALGARSLLSSPRSYLLLLYFMPLALLSAFSHQEARFLIPLLVPLVLLCSPQTQPVPWKGTVVLFNALGALLFGCLHQGGLVPGLEYLEQVVHAPVLPSTPTHYTLLFTHTYMPPRHLLHLPGLGAPVEVVDMGGTEDWALCQTLKSFTRQPACQVAGGPWLCRLFVVTPGTTRRAVEKCSFPFKNETLLFPHLTLEDPPALSSLLSGAWRDHLSLHIVELGEET",
# VRK2_HUMAN
"MPPKRNEKYKLPIPFPEGKVLDDMEGNQWVLGKKIGSGGFGLIYLAFPTNKPEKDARHVVKVEYQENGPLFSELKFYQRVAKKDCIKKWIERKQLDYLGIPLFYGSGLTEFKGRSYRFMVMERLGIDLQKISGQNGTFKKSTVLQLGIRMLDVLEYIHENEYVHGDIKAANLLLGYKNPDQVYLADYGLSYRYCPNGNHKQYQENPRKGHNGTIEFTSLDAHKGVALSRRSDVEILGYCMLRWLCGKLPWEQNLKDPVAVQTAKTNLLDELPQSVLKWAPSGSSCCEIAQFLVCAHSLAYDEKPNYQALKKILNPHGIPLGPLDFSTKGQSINVHTPNSQKVDSQKAATKQVNKAHNRLIEKKVHSERSAESCATWKVQKEEKLIGLMNNEAAQESTRRRQKYQESQEPLNEVNSFPQKISYTQFPNSFYEPHQDFTSPDIFKKSRSPSWYKYTSTVSTGITDLESSTGLWPTISQFTLSEETNADVYYYRIIIPVLLMLVFLALFFL",
# ERO1B_HUMAN
"MSQGVRRAGAGQGVAAAVQLLVTLSFLRSVVEAQVTGVLDDCLCDIDSIDNFNTYKIFPKIKKLQERDYFRYYKVNLKRPCPFWAEDGHCSIKDCHVEPCPESKIPVGIKAGHSNKYLKMANNTKELEDCEQANKLGAINSTLSNQSKEAFIDWARYDDSRDHFCELDDERSPAAQYVDLLLNPERYTGYKGTSAWRVWNSIYEENCFKPRSVYRPLNPLAPSRGEDDGESFYTWLEGLCLEKRVFYKLISGLHASINLHLCANYLLEETWGKPSWGPNIKEFKHRFDPVETKGEGPRRLKNLYFLYLIELRALSKVAPYFERSIVDLYTGNAEEDADTKTLLLNIFQDTKSFPMHFDEKSMFAGDKKGAKSLKEEFRLHFKNISRIMDCVGCDKCRLWGKLQTQGLGTALKILFSEKEIQKLPENSPSKGFQLTRQEIVALLNAFGRLSTSIRDLQNFKVLLQHSR",
# LRC33_HUMAN
"MELLPLWLCLGFHFLTVGWRNRSGTATAASQGVCKLVGGAADCRGQSLASVPSSLPPHARMLTLDANPLKTLWNHSLQPYPLLESLSLHSCHLERISRGAFQEQGHLRSLVLGDNCLSENYEETAAALHALPGLRRLDLSGNALTEDMAALMLQNLSSLRSVSLAGNTIMRLDDSVFEGLERLRELDLQRNYIFEIEGGAFDGLAELRHLNLAFNNLPCIVDFGLTRLRVLNVSYNVLEWFLATGGEAAFELETLDLSHNQLLFFPLLPQYSKLRTLLLRDNNMGFYRDLYNTSSPREMVAQFLLVDGNVTNITTVSLWEEFSSSDLADLRFLDMSQNQFQYLPDGFLRKMPSLSHLNLHQNCLMTLHIREHEPPGALTELDLSHNQLSELHLAPGLASCLGSLRLFNLSSNQLLGVPPGLFANARNITTLDMSHNQISLCPLPAASDRVGPPSCVDFRNMASLRSLSLEGCGLGALPDCPFQGTSLTYLDLSSNWGVLNGSLAPLQDVAPMLQVLSLRNMGLHSSFMALDFSGFGNLRDLDLSGNCLTTFPRFGGSLALETLDLRRNSLTALPQKAVSEQLSRGLRTIYLSQNPYDCCGVDGWGALQHGQTVADWAMVTCNLSSKIIRVTELPGGVPRDCKWERLDLGLLYLVLILPSCLTLLVACTVIVLTFKKPLLQVIKSRCHWSSVY",
# TMC8_HUMAN
"MLLPRSVSSERAPGVPEPEELWEAEMERLRGSGTPVRGLPYAMMDKRLIWQLREPAGVQTLRWQRWQRRRQTVERRLREAAQRLARGLGLWEGALYEIGGLFGTGIRSYFTFLRFLLLLNLLSLLLTASFVLLPLVWLRPPDPGPTLNLTLQCPGSRQSPPGVLRFHNQLWHVLTGRAFTNTYLFYGAYRVGPESSSVYSIRLAYLLSPLACLLLCFCGTLRRMVKGLPQKTLLGQGYQAPLSAKVFSSWDFCIRVQEAATIKKHEISNEFKVELEEGRRFQLMQQQTRAQTACRLLSYLRVNVLNGLLVVGAISAIFWATKYSQDNKEESLFLLLQYLPPGVIALVNFLGPLLFTFLVQLENYPPNTEVNLTLIWCVVLKLASLGMFSVSLGQTILCIGRDKSSCESYGYNVCDYQCWENSVGEELYKLSIFNFLLTVAFAFLVTLPRRLLVDRFSGRFWAWLEREEFLVPKNVLDIVAGQTVTWMGLFYCPLLPLLNSVFLFLTFYIKKYTLLKNSRASSRPFRASSSTFFFQLVLLLGLLLAAVPLGYVVSSIHSSWDCGLFTNYSAPWQVVPELVALGLPPIGQRALHYLGSHAFSFPLLIMLSLVLTVCVSQTQANARAIHRLRKQLVWQVQEKWHLVEDLSRLLPEPGPSDSPGPKYPASQASRPQSFCPGCPCPGSPGHQAPRPGPSVVDAAGLRSPCPGQHGAPASARRFRFPSGAEL",
# CERS3_HUMAN
"MFWTFKEWFWLERFWLPPTIKWSDLEDHDGLVFVKPSHLYVTIPYAFLLLIIRRVFEKFVASPLAKSFGIKETVRKVTPNTVLENFFKHSTRQPLQTDIYGLAKKCNLTERQVERWFRSRRNQERPSRLKKFQEACWRFAFYLMITVAGIAFLYDKPWLYDLWEVWNGYPKQPLLPSQYWYYILEMSFYWSLLFRLGFDVKRKDFLAHIIHHLAAISLMSFSWCANYIRSGTLVMIVHDVADIWLESAKMFSYAGWTQTCNTLFFIFSTIFFISRLIVFPFWILYCTLILPMYHLEPFFSYIFLNLQLMILQVLHLYWGYYILKMLNRCIFMKSIQDVRSDDEDYEEEEEEEEEEATKGKEMDCLKNGLRAERHLIPNGQHGH",
# KSR1_HUMAN
"MDRAALRAAAMGEKKEGGGGGDAAAAEGGAGAAASRALQQCGQLQKLIDISIGSLRGLRTKCAVSNDLTQQEIRTLEAKLVRYICKQRQCKLSVAPGERTPELNSYPRFSDWLYTFNVRPEVVQEIPRDLTLDALLEMNEAKVKETLRRCGASGDECGRLQYALTCLRKVTGLGGEHKEDSSWSSLDARRESGSGPSTDTLSAASLPWPPGSSQLGRAGNSAQGPRSISVSALPASDSPTPSFSEGLSDTCIPLHASGRLTPRALHSFITPPTTPQLRRHTKLKPPRTPPPPSRKVFQLLPSFPTLTRSKSHESQLGNRIDDVSSMRFDLSHGSPQMVRRDIGLSVTHRFSTKSWLSQVCHVCQKSMIFGVKCKHCRLKCHNKCTKEAPACRISFLPLTRLRRTESVPSDINNPVDRAAEPHFGTLPKALTKKEHPPAMNHLDSSSNPSSTTSSTPSSPAPFPTSSNPSSATTPPNPSPGQRDSRFNFPAAYFIHHRQQFIFPVPSAGHCWKCLLIAESLKENAFNISAFAHAAPLPEAADGTRLDDQPKADVLEAHEAEAEEPEAGKSEAEDDEDEVDDLPSSRRPWRGPISRKASQTSVYLQEWDIPFEQVELGEPIGQGRWGRVHRGRWHGEVAIRLLEMDGHNQDHLKLFKKEVMNYRQTRHENVVLFMGACMNPPHLAIITSFCKGRTLHSFVRDPKTSLDINKTRQIAQEIIKGMGYLHAKGIVHKDLKSKNVFYDNGKVVITDFGLFGISGVVREGRRENQLKLSHDWLCYLAPEIVREMTPGKDEDQLPFSKAADVYAFGTVWYELQARDWPLKNQAAEASIWQIGSGEGMKRVLTSVSLGKEVSEILSACWAFDLQERPSFSLLMDMLEKLPKLNRRLSHPGHFWKSADINSSKVVPRFERFGLGVLESSNPKM",
# TEX2_HUMAN
"MTSLYGRHAEKTTDMPKPSAPKVHVQRSVSRDTIAIHFSASGEEEEEEEEEFREYFEEGLDDQSIVTGLEAKEDLYLEPQVGHDPAGPAASPVLADGLSVSQAPAILPVSKNTVKLLESPVPAAQVLSTVPLAVSPGSSSSGPLASSPSVSSLSEQKTSSSSPLSSPSKSPILSSSASTSTLSSAKPFMSLVKSLSTEVEPKESPHPARHRHLMKTLVKSLSTDTSRQESDTVSYKPPDSKLNLHLFKQFTQPRNTGGDSKTAPSSPLTSPSDTRSFFKVPEMEAKIEDTKRRLSEVIYEPFQLLSKIIGEESGSHRPKALSSSASELSNLSSLNGHLESNNNYSIKEEECDSEGDGYGSDSNIPRSDHPKSTGEPTREIELKSSQGSSLKDLGLKTSSLVLEKCSLSALVSKEDEEFCELYTEDFDLETEGESKVDKLSDIPLKPEVLAEDGVVLDSEDEVDSAVQHPELPVKTLGFFIMCVYVYLILPLPHYVSGLFLGIGLGFMTAVCVIWFFTPPSAHKYHKLHKNLRHWNTRSLDIKEPEILKGWMNEIYNYDPETYHATLTHSVFVRLEGGTLRLSKPNKNISRRASYNEPKPEVTYISQKIYDLSDSKIYLVPKTLARKRIWNKKYPICIELGQQDDFMSKAQTDKETSEEKPPAEGSEDPKKPPRPQEGTRSSQRDQILYLFGRTGREKEEWFRRFILASKLKSEIKKSSGVSGGKPGLLPAHSRHNSPSGHLTHSRSSSKGSVEEIMSQPKQKELAGSVRQKMLLDYSVYMGRCVPQESRSPQRSPLQSAESSPTAGKKLPEVPPSEEEEQEAWVNALLGRIFWDFLGEKYWSDLVSKKIQMKLSKIKLPYFMNELTLTELDMGVAVPKILQAFKPYVDHQGLWIDLEMSYNGSFLMTLETKMNLTKLGKEPLVEALKVGEIGKEGCRPRAFCLADSDEESSSAGSSEEDDAPEPSGGDKQLLPGAEGYVGGHRTSKIMRFVDKITKSKYFQKATETEFIKKKIEEVSNTPLLLTVEVQECRGTLAVNIPPPPTDRVWYGFRKPPHVELKARPKLGEREVTLVHVTDWIEKKLEQEFQKVFVMPNMDDVYITIMHSAMDPRSTSCLLKDPPVEAADQP",
# TRI59_HUMAN
"MHNFEEELTCPICYSIFEDPRVLPCSHTFCRNCLENILQASGNFYIWRPLRIPLKCPNCRSITEIAPTGIESLPVNFALRAIIEKYQQEDHPDIVTCPEHYRQPLNVYCLLDKKLVCGHCLTIGQHHGHPIDDLQSAYLKEKDTPQKLLEQLTDTHWTDLTHLIEKLKEQKSHSEKMIQGDKEAVLQYFKELNDTLEQKKKSFLTALCDVGNLINQEYTPQIERMKEIREQQLELMALTISLQEESPLKFLEKVDDVRQHVQILKQRPLPEVQPVEIYPRVSKILKEEWSRTEIGQIKNVLIPKMKISPKRMSCSWPGKDEKEVEFLKILNIVVVTLISVILMSILFFNQHIITFLSEITLIWFSEASLSVYQSLSNSLHKVKNILCHIFYLLKEFVWKIVSH",
# SGPP2_HUMAN
"MAELLRSLQDSQLVARFQRRCGLFPAPDEGPRENGADPTERAARVPGVEHLPAANGKGGEAPANGLRRAAAPEAYVQKYVVKNYFYYYLFQFSAALGQEVFYITFLPFTHWNIDPYLSRRLIIIWVLVMYIGQVAKDVLKWPRPSSPPVVKLEKRLIAEYGMPSTHAMAATAIAFTLLISTMDRYQYPFVLGLVMAVVFSTLVCLSRLYTGMHTVLDVLGGVLITALLIVLTYPAWTFIDCLDSASPLFPVCVIVVPFFLCYNYPVSDYYSPTRADTTTILAAGAGVTIGFWINHFFQLVSKPAESLPVIQNIPPLTTYMLVLGLTKFAVGIVLILLVRQLVQNLSLQVLYSWFKVVTRNKEARRRLEIEVPYKFVTYTSVGICATTFVPMLHRFLGLP",
# PLPL6_HUMAN
"MEAPLQTGMMGTSSHGLATNSSGAKVAERDGFQDVLAPGEGSAGRICGAQPVPFVPQVLGVMIGAGVAVVVTAVLILLVVRRLRVPKTPAPDGPRYRFRKRDKVLFYGRKIMRKVSQSTSSLVDTSVSATSRPRMRKKLKMLNIAKKILRIQKETPTLQRKEPPPAVLEADLTEGDLANSHLPSEVLYMLKNVRVLGHFEKPLFLELCRHMVFQRLGQGDYVFRPGQPDASIYVVQDGLLELCLPGPDGKECVVKEVVPGDSVNSLLSILDVITGHQHPQRTVSARAARDSTVLRLPVEAFSAVFTKYPESLVRVVQIIMVRLQRVTFLALHNYLGLTNELFSHEIQPLRLFPSPGLPTRTSPVRGSKRMVSTSATDEPRETPGRPPDPTGAPLPGPTGDPVKPTSLETPSAPLLSRCVSMPGDISGLQGGPRSDFDMAYERGRISVSLQEEASGGSLAAPARTPTQEPREQPAGACEYSYCEDESATGGCPFGPYQGRQTSSIFEAAKQELAKLMRIEDPSLLNSRVLLHHAKAGTIIARQGDQDVSLHFVLWGCLHVYQRMIDKAEDVCLFVAQPGELVGQLAVLTGEPLIFTLRAQRDCTFLRISKSDFYEIMRAQPSVVLSAAHTVAARMSPFVRQMDFAIDWTAVEAGRALYRQGDRSDCTYIVLNGRLRSVIQRGSGKKELVGEYGRGDLIGVVEALTRQPRATTVHAVRDTELAKLPEGTLGHIKRRYPQVVTRLIHLLSQKILGNLQQLQGPFPAGSGLGVPPHSELTNPASNLATVAILPVCAEVPMVAFTLELQHALQAIGPTLLLNSDIIRARLGASALDSIQEFRLSGWLAQQEDAHRIVLYQTDASLTPWTVRCLRQADCILIVGLGDQEPTLGQLEQMLENTAVRALKQLVLLHREEGAGPTRTVEWLNMRSWCSGHLHLRCPRRLFSRRSPAKLHELYEKVFSRRADRHSDFSRLARVLTGNTIALVLGGGGARGCSHIGVLKALEEAGVPVDLVGGTSIGSFIGALYAEERSASRTKQRAREWAKSMTSVLEPVLDLTYPVTSMFTGSAFNRSIHRVFQDKQIEDLWLPYFNVTTDITASAMRVHKDGSLWRYVRASMTLSGYLPPLCDPKDGHLLMDGGYINNLPADIARSMGAKTVIAIDVGSQDETDLSTYGDSLSGWWLLWKRLNPWADKVKVPDMAEIQSRLAYVSCVRQLEVVKSSSYCEYLRPPIDCFKTMDFGKFDQIYDVGYQYGKAVFGGWSRGNVIEKMLTDRRSTDLNESRRADVLAFPSSGFTDLAEIVSRIEPPTSYVSDGCADGEESDCLTEYEEDAGPDCSRDEGGSPEGASPSTASEMEEEKSILRQRRCLPQEPPGSATDA",
# ZDH14_HUMAN
"MPPGGGGPMKDCEYSQISTHSSSPMESPHKKKKIAARRKWEVFPGRNKFFCNGRIMMARQTGVFYLTLVLILVTSGLFFAFDCPYLAVKITPAIPAVAGILFFFVMGTLLRTSFSDPGVLPRATPDEAADLERQIDIANGTSSGGYRPPPRTKEVIINGQTVKLKYCFTCKIFRPPRASHCSLCDNCVERFDHHCPWVGNCVGKRNYRFFYMFILSLSFLTVFIFAFVITHVILRSQQTGFLNALKDSPASVLEAVVCFFSVWSIVGLSGFHTYLISSNQTTNEDIKGSWSNKRGKENYNPYSYGNIFTNCCVALCGPISPSLIDRRGYIQPDTPQPAAPSNGITMYGATQSQSDMCDQDQCIQSTKFVLQAAATPLLQSEPSLTSDELHLPGKPGLGTPCASLTLGPPTPPASMPNLAEATLADVMPRKDEHMGHQFLTPDEAPSPPRLLAAGSPLAHSRTMHVLGLASQDSLHEDSVRGLVKLSSV",
# VKORL_HUMAN
"MAAPVLLRVSVPRWERVARYAVCAAGILLSIYAYHVEREKERDPEHRALCDLGPWVKCSAALASRWGRGFGLLGSIFGKDGVLNQPNSVFGLIFYILQLLLGMTASAVAALILMTSSIMSVVGSLYLAYILYFVLKEFCIICIVTYVLNFLLLIINYKRLVYLNEAWKRQLQPKQD",
# ORML3_HUMAN
"MNVGTAHSEVNPNTRVMNSRGIWLSYVLAIGLLHIVLLSIPFVSVPVVWTLTNLIHNMGMYIFLHTVKGTPFETPDQGKARLLTHWEQMDYGVQFTASRKFLTITPIVLYFLTSFYTKYDQIHFVLNTVSLMSVLIPKLPQLHGVRIFGINKY",
# ARM10_HUMAN
"MGGPRGAGWVAAGLLLGAGACYCIYRLTRGRRRGDRELGIRSSKSAGALEEGTSEGQLCGRSARPQTGGTWESQWSKTSQPEDLTDGSYDDVLNAEQLQKLLYLLESTEDPVIIERALITLGNNAAFSVNQAIIRELGGIPIVANKINHSNQSIKEKALNALNNLSVNVENQIKIKIYISQVCEDVFSGPLNSAVQLAGLTLLTNMTVTNDHQHMLHSYITDLFQVLLTGNGNTKVQVLKLLLNLSENPAMTEGLLRAQVDSSFLSLYDSHVAKEILLRVLTLFQNIKNCLKIEGHLAVQPTFTEGSLFFLLHGEECAQKIRALVDHHDAEVKEKVVTIIPKI",
# UB2J2_HUMAN
"MSSTSSKRAPTTATQRLKQDYLRIKKDPVPYICAEPLPSNILEWHYVVRGPEMTPYEGGYYHGKLIFPREFPFKPPSIYMITPNGRFKCNTRLCLSITDFHPDTWNPAWSVSTILTGLLSFMVEKGPTLGSIETSDFTKRQLAVQSLAFNLKDKVFCELFPEVVEEIKQKQKAQDELSSRPQTLPLPDVVPDGETHLVQNGIQLLNGHAPGAVPNLAGLQQANRHHGLLGGALANLFVIVGFAAFAYTVKYVLRSIAQE",
# TMTC2_HUMAN
"MIAELVSSALGLALYLNTLSADFCYDDSRAIKTNQDLLPETPWTHIFYNDFWGTLLTHSGSHKSYRPLCTLSFRLNHAIGGLNPWSYHLVNVLLHAAVTGLFTSFSKILLGDGYWTFMAGLMFASHPIHTEAVAGIVGRADVGASLFFLLSLLCYIKHCSTRGYSARTWGWFLGSGLCAGCSMLWKEQGVTVLAVSAVYDVFVFHRLKIKQILPTIYKRKNLSLFLSISLLIFWGSSLLGARLYWMGNKPPSFSNSDNPAADSDSLLTRTLTFFYLPTKNLWLLLCPDTLSFDWSMDAVPLLKTVCDWRNLHTVAFYTGLLLLAYYGLKSPSVDRECNGKTVTNGKQNANGHSCLSDVEYQNSETKSSFASKVENGIKNDVSQRTQLPSTENIVVLSLSLLIIPFVPATNLFFYVGFVIAERVLYIPSMGFCLLITVGARALYVKVQKRFLKSLIFYATATLIVFYGLKTAIRNGDWQNEEMLYRSGIKVNPAKAWGNLGNVLKSQSKISEAESAYRNALYYRSNMADMLYNLGLLLQENSRFAEALHYYKLAIGSRPTLASAYLNTGIILMNQGRTEEARRTFLKCSEIPDENLKDPHAHKSSVTSCLYNLGKLYHEQGHYEEALSVYKEAIQKMPRQFAPQSLYNMMGEAYMRLSKLPEAEHWYMESLRSKTDHIPAHLTYGKLLALTGRKSEAEKLFLKAIELDPTKGNCYMHYGQFLLEEARLIEAAEMAKKAAELDSTEFDVVFNAAHMLRQASLNEAAEKYYDLAARLRPNYPAALMNLGAILHLNGRLQKAEANYLRALQLKPDDVITQSNLRKLWNIMEKQGLKTSKT",
# RDHE2_HUMAN
"MSFNLQSSKKLFIFLGKSLFSLLEAMIFALLPKPRKNVAGEIVLITGAGSGLGRLLALQFARLGSVLVLWDINKEGNEETCKMAREAGATRVHAYTCDCSQKEGVYRVADQVKKEVGDVSILINNAGIVTGKKFLDCPDELMEKSFDVNFKAHLWTYKAFLPAMIANDHGHLVCISSSAGLSGVNGLADYCASKFAAFGFAESVFVETFVQKQKGIKTTIVCPFFIKTGMFEGCTTGCPSLLPILEPKYAVEKIVEAILQEKMYLYMPKLLYFMMFLKSFLPLKTGLLIADYLGILHAMDGFVDQKKKL",
# VMA12_HUMAN
"MASSLLAGERLVRALGPGGELEPERLPRKLRAELEAALGKKHKGGDSSSGPQRLVSFRLIRDLHQHLRERDSKLYLHELLEGSEIYLPEVVKPPRNPELVARLEKIKIQLANEEYKRITRNVTCQDTRHGGTLSDLGKQVRSLKALVITIFNFIVTVVAAFVCTYLGSQYIFTEMASRVLAALIVASVVGLAELYVMVRAMEGELGEL",
# CERS5_HUMAN
"MATAAQGPLSLLWGWLWSERFWLPENVSWADLEGPADGYGYPRGRHILSVFPLAAGIFFVRLLFERFIAKPCALCIGIEDSGPYQAQPNAILEKVFISITKYPDKKRLEGLSKQLDWNVRKIQCWFRHRRNQDKPPTLTKFCESMWRFTFYLCIFCYGIRFLWSSPWFWDIRQCWHNYPFQPLSSGLYHYYIMELAFYWSLMFSQFTDIKRKDFLIMFVHHLVTIGLISFSYINNMVRVGTLIMCLHDVSDFLLEAAKLANYAKYQRLCDTLFVIFSAVFMVTRLGIYPFWILNTTLFESWEIIGPYASWWLLNGLLLTLQLLHVIWSYLIARIALKALIRGKVSKDDRSDVESSSEEEDVTTCTKSPCDSSSSNGANRVNGHMGGSYWAEE",
# CISD2_HUMAN
"MVLESVARIVKVQLPAYLKRLPVPESITGFARLTVSEWLRLLPFLGVLALLGYLAVRPFLPKKKQQKDSLINLKIQKENPKVVNEINIEDLCLTKAAYCRCWRSKTFPACDGSHNKHNELTGDNVGPLILKKKEV",
# JAGN1_HUMAN
"MASRAGPRAAGTDGSDFQHRERVAMHYQMSVTLKYEIKKLIYVHLVIWLLLVAKMSVGHLRLLSHDQVAMPYQWEYPYLLSILPSLLGLLSFPRNNISYLVLSMISMGLFSIAPLIYGSMEMFPAAQQLYRHGKAYRFLFGFSAVSIMYLVLVLAVQVHAWQLYYSKKLLDSWFTSTQEKKHK",
# PAR16_HUMAN
"MQPSGWAAAREAAGRDMLAADLRCSLFASALQSYKRDSVLRPFPASYARGDCKDFEALLADASKLPNLKELLQSSGDNHKRAWDLVSWILSSKVLTIHSAGKAEFEKIQKLTGAPHTPVPAPDFLFEIEYFDPANAKFYETKGERDLIYAFHGSRLENFHSIIHNGLHCHLNKTSLFGEGTYLTSDLSLALIYSPHGHGWQHSLLGPILSCVAVCEVIDHPDVKCQTKKKDSKEIDRRRARIKHSEGGDIPPKYFVVTNNQLLRVKYLLVYSQKPPKRASSQLSWFSSHWFTVMISLYLLLLLIVSVINSSAFQHFWNRAKR",
# SERP2_HUMAN
"MVAKQRIRMANEKHSKNITQRGNVAKTLRPQEEKYPVGPWLLALFVFVVCGSAIFQIIQSIRMGM",
# AN13C_HUMAN
"MTGEKIRSLRRDHKPSKEEGDLLEPGDEEAAAALGGTFTRSRIGKGGKACHKIFSNHHHRLQLKAAPASSNPPGAPALPLHNSSVTANSQSPALLAGTNPVAVVADGGSCPAHYPVHECVFKGDVRRLSSLIRTHNIGQKDNHGNTPLHLAVMLGNKECAHLLLAHNAPVKVKNAQGWSPLAEAISYGDRQMITALLRKLKQQSRESVEEKRPRLLKALKELGDFYLELHWDFQSWVPLLSRILPSDACKIYKQGINIRLDTTLIDFTDMKCQRGDLSFIFNGDAAPSESFVVLDNEQKVYQRIHHEESEMETEEEVDILMSSDIYSATLSTKSISFTRAQTGWLFREDKTERVGNFLADFYLVNGLVLESRKRREHLSEEDILRNKAIMESLSKGGNIMEQNFEPIRRQSLTPPPQNTITWEEYISAENGKAPHLGRELVCKESKKTFKATIAMSQEFPLGIELLLNVLEVVAPFKHFNKLREFVQMKLPPGFPVKLDIPVFPTITATVTFQEFRYDEFDGSIFTIPDDYKEDPSRFPDL",
# ZDH22_HUMAN
"MLALRLLNVVAPAYFLCISLVTFVLQLFLFLPSMREDPAAARLFSPALLHGALFLFLSANALGNYVLVIQNSPDDLGACQGASARKTPCPSPSTHFCRVCARVTLRHDHHCFFTGNCIGSRNMRNFVLFCLYTSLACLYSMVAGVAYISAVLSISFAHPLAFLTLLPTSISQFFSGAVLGSEMFVILMLYLWFAIGLACAGFCCHQLLLILRGQTRHQVRKGVAVRARPWRKNLQEVFGKRWLLGLLVPMFNVGSESSKQQDK",
# PNDC1_HUMAN
"MFCTRGLLFFAFLAGLDIEFTGLRSNLSGPQQISLFDLPSEWYLKTRQSVQQFTVCQIGLSVFSAIEGEANKYIAHSCNFYLFPTTFGILDSEFSFQASSVQFLNQYGFNYNKFLKNGIPYMNEEQEKKIRHDILTGNWRVRSSPDKDQIKVVIDEVTRWLELAKEGDWMTLPGITGFQAFEVQLVLRQALPNIWTVLKDEGVVVKKVSKQHRWYLQNTSCDRESCWKENILLSARGFSVFFQMLVKAQKPLVGHNMMMDLLHLHEKFFRPLPESYDQFKQNIHSLFPVLIDTKSVTKDIWKEMNFPRVSNLSEVYEVLNSDLNPTKNSGPEIVHASRCEKYVETKCPHEAAYDAFLCGSVLLKVAHLLLQKIYHIDPVPESSFPQYLDVLAPYVNQVNLIRAGVPKINFSGPDYPSIRPPILILSVKRWPGVSEQQVYHKFQNLCKFDVRRLTRSQFLLLTNKFKDARNILKEYRDHPTLCISLYRYWRHSPNVNCLLQVCGIVTAWALLAFILGRSGT",
# PMGT2_HUMAN
"MHLSAVFNALLVSVLAAVLWKHVRLREHAATLEEELALSRQATEPAPALRIDYPKALQILMEGGTHMVCTGRTHTDRICRFKWLCYSNEAEEFIFFHGNTSVMLPNLGSRRFQPALLDLSTVEDHNTQYFNFVELPAAALRFMPKPVFVPDVALIANRFNPDNLMHVFHDDLLPLFYTLRQFPGLAHEARLFFMEGWGEGAHFDLYKLLSPKQPLLRAQLKTLGRLLCFSHAFVGLSKITTWYQYGFVQPQGPKANILVSGNEIRQFARFMTEKLNVSHTGVPLGEEYILVFSRTQNRLILNEAELLLALAQEFQMKTVTVSLEDHTFADVVRLVSNASMLVSMHGAQLVTTLFLPRGATVVELFPYAVNPDHYTPYKTLAMLPGMDLQYVAWRNMMPENTVTHPERPWDQGGITHLDRAEQARILQSREVPRHLCCRNPEWLFRIYQDTKVDIPSLIQTIRRVVKGRPGPRKQKWTVGLYPGKVREARCQASVHGASEARLTVSWQIPWNLKYLKVREVKYEVWLQEQGENTYVPYILALQNHTFTENIKPFTTYLVWVRCIFNKILLGPFADVLVCNT",
# AT11C_HUMAN
"MQMVPSLPPASECAGEEKRVGTRTVFVGNHPVSETEAYIAQRFCDNRIVSSKYTLWNFLPKNLFEQFRRIANFYFLIIFLVQVTVDTPTSPVTSGLPLFFVITVTAIKQGYEDCLRHRADNEVNKSTVYIIENAKRVRKESEKIKVGDVVEVQADETFPCDLILLSSCTTDGTCYVTTASLDGESNCKTHYAVRDTIALCTAESIDTLRAAIECEQPQPDLYKFVGRINIYSNSLEAVARSLGPENLLLKGATLKNTEKIYGVAVYTGMETKMALNYQGKSQKRSAVEKSINAFLIVYLFILLTKAAVCTTLKYVWQSTPYNDEPWYNQKTQKERETLKVLKMFTDFLSFMVLFNFIIPVSMYVTVEMQKFLGSFFISWDKDFYDEEINEGALVNTSDLNEELGQVDYVFTDKTGTLTENSMEFIECCIDGHKYKGVTQEVDGLSQTDGTLTYFDKVDKNREELFLRALCLCHTVEIKTNDAVDGATESAELTYISSSPDEIALVKGAKRYGFTFLGNRNGYMRVENQRKEIEEYELLHTLNFDAVRRRMSVIVKTQEGDILLFCKGADSAVFPRVQNHEIELTKVHVERNAMDGYRTLCVAFKEIAPDDYERINRQLIEAKMALQDREEKMEKVFDDIETNMNLIGATAVEDKLQDQAAETIEALHAAGLKVWVLTGDKMETAKSTCYACRLFQTNTELLELTTKTIEESERKEDRLHELLIEYRKKLLHEFPKSTRSFKKAWTEHQEYGLIIDGSTLSLILNSSQDSSSNNYKSIFLQICMKCTAVLCCRMAPLQKAQIVRMVKNLKGSPITLSIGDGANDVSMILESHVGIGIKGKEGRQAARNSDYSVPKFKHLKKLLLAHGHLYYVRIAHLVQYFFYKNLCFILPQFLYQFFCGFSQQPLYDAAYLTMYNICFTSLPILAYSLLEQHINIDTLTSDPRLYMKISGNAMLQLGPFLYWTFLAAFEGTVFFFGTYFLFQTASLEENGKVYGNWTFGTIVFTVLVFTVTLKLALDTRFWTWINHFVIWGSLAFYVFFSFFWGGIIWPFLKQQRMYFVFAQMLSSVSTWLAIILLIFISLFPEILLIVLKNVRRRSARRNLSCRRASDSLSARPSVRPLLLRTFSDESNVL",
# UBAC2_HUMAN
"MFTSTGSSGLYKAPLSKSLLLVPSALSLLLALLLPHCQKLFVYDLHAVKNDFQIWRLICGRIICLDLKDTFCSSLLIYNFRIFERRYGSRKFASFLLGSWVLSALFDFLLIEAMQYFFGITAASNLPSGFLAPVFALFVPFYCSIPRVQVAQILGPLSITNKTLIYILGLQLFTSGSYIWIVAISGLMSGLCYDSKMFQVHQVLCIPSWMAKFFSWTLEPIFSSSEPTSEARIGMGATLDIQRQQRMELLDRQLMFSQFAQGRRQRQQQGGMINWNRLFPPLRQRQNVNYQGGRQSEPAAPPLEVSEEQVARLMEMGFSRGDALEALRASNNDLNVATNFLLQH",
# AQP11_HUMAN
"MSPLLGLRSELQDTCTSLGLMLSVVLLMGLARVVARQQLHRPVAHAFVLEFLATFQLCCCTHELQLLSEQHPAHPTWTLTLVYFFSLVHGLTLVGTSSNPCGVMMQMMLGGMSPETGAVRLLAQLVSALCSRYCTSALWSLGLTQYHVSERSFACKNPIRVDLLKAVITEAVCSFLFHSALLHFQEVRTKLRIHLLAALITFLVYAGGSLTGAVFNPALALSLHFMCFDEAFPQFFIVYWLAPSLGILLMILMFSFFLPWLHNNHTINKKE",
# B3GL2_HUMAN
"MRNWLVLLCPCVLGAALHLWLRLRSPPPACASGAGPADQLALFPQWKSTHYDVVVGVLSARNNHELRNVIRSTWMRHLLQHPTLSQRVLVKFIIGAHGCEVPVEDREDPYSCKLLNITNPVLNQEIEAFSLSEDTSSGLPEDRVVSVSFRVLYPIVITSLGVFYDANDVGFQRNITVKLYQAEQEEALFIARFSPPSCGVQVNKLWYKPVEQFILPESFEGTIVWESQDLHGLVSRNLHKVTVNDGGGVLRVITAGEGALPHEFLEGVEGVAGGFIYTIQEGDALLHNLHSRPQRLIDHIRNLHEEDALLKEESSIYDDIVFVDVVDTYRNVPAKLLNFYRWTVETTSFNLLLKTDDDCYIDLEAVFNRIVQKNLDGPNFWWGNFRLNWAVDRTGKWQELEYPSPAYPAFACGSGYVISKDIVKWLASNSGRLKTYQGEDVSMGIWMAAIGPKRYQDSLWLCEKTCETGMLSSPQYSPWELTELWKLKERCGDPCRCQAR",
# PCAT1_HUMAN
"MRLRGCGPRAAPASSAGASDARLLAPPGRNPFVHELRLSALQKAQVALMTLTLFPVRLLVAAAMMLLAWPLALVASLGSAEKEPEQPPALWRKVVDFLLKAIMRTMWFAGGFHRVAVKGRQALPTEAAILTLAPHSSYFDAIPVTMTMSSIVMKAESRDIPIWGTLIQYIRPVFVSRSDQDSRRKTVEEIKRRAQSNGKWPQIMIFPEGTCTNRTCLITFKPGAFIPGAPVQPVVLRYPNKLDTITWTWQGPGALEILWLTLCQFHNQVEIEFLPVYSPSEEEKRNPALYASNVRRVMAEALGVSVTDYTFEDCQLALAEGQLRLPADTCLLEFARLVRGLGLKPEKLEKDLDRYSERARMKGGEKIGIAEFAASLEVPVSDLLEDMFSLFDESGSGEVDLRECVVALSVVCRPARTLDTIQLAFKMYGAQEDGSVGEGDLSCILKTALGVAELTVTDLFRAIDQEEKGKITFADFHRFAEMYPAFAEEYLYPDQTHFESCAETSPAPIPNGFCADFSPENSDAGRKPVRKKLD",
# TOIP2_HUMAN
"MADSGLREPQEDSQKDLENDPSVNSQAQETTIIASNAEEAEILHSACGLSKDHQEVETEGPESADTGDKSESPDEANVGKHPKDKTEDENKQSFLDGGKGHHLPSENLGKEPLDPDPSHSPSDKVGRADAHLGSSSVALPKEASDGTGASQEPPTTDSQEAQSPGHSSAGQEGEDTLRRRLLAPEAGSHPQQTQKLEEIKENAQDTMRQINKKGFWSYGPVILVVLVVAVVASSVNSYYSSPAQQVPKNPALEAFLAQFSQLEDKFPGQSSFLWQRGRKFLQKHLNASNPTEPATIIFTAAREGRETLKCLSHHVADAYTSSQKVSPIQIDGAGRTWQDSDTVKLLVDLELSYGFENGQKAAVVHHFESFPAGSTLIFYKYCDHENAAFKDVALVLTVLLEEETLEASVGPRETEEKVRDLLWAKFTNSDTPTSFNHMDSDKLSGLWSRISHLVLPVQPVSSIEEQGCLF",
# ATLA2_HUMAN
"MAEGDEAARGQQPHQGLWRRRRTSDPSAAVNHVSSTTSLGENYEDDDLVNSDEVMKKPCPVQIVLAHEDDHNFELDEEALEQILLQEHIRDLNIVVVSVAGAFRKGKSFLLDFMLRYMYNKDSQSWIGGNNEPLTGFTWRGGCERETTGIQVWNEVFVIDRPNGTKVAVLLMDTQGAFDSQSTIKDCATVFALSTMTSSVQVYNLSQNIQEDDLQHLQLFTEYGRLAMEEIYQKPFQTLMFLIRDWSYPYEHSYGLEGGKQFLEKRLQVKQNQHEELQNVRKHIHNCFSNLGCFLLPHPGLKVATNPSFDGRLKDIDEDFKRELRNLVPLLLAPENLVEKEISGSKVTCRDLVEYFKAYIKIYQGEELPHPKSMLQATAEANNLAAVAGARDTYCKSMEQVCGGDKPYIAPSDLERKHLDLKEVAIKQFRSVKKMGGDEFCRRYQDQLEAEIEETYANFIKHNDGKNIFYAARTPATLFAVMFAMYIISGLTGFIGLNSIAVLCNLVMGLALIFLCTWAYVKYSGEFREIGTVIDQIAETLWEQVLKPLGDNLMEENIRQSVTNSIKAGLTDQVSHHARLKTD",
# NPL4_HUMAN
"MAESIIIRVQSPDGVKRITATKRETAATFLKKVAKEFGFQNNGFSVYINRNKTGEITASSNKSLNLLKIKHGDLLFLFPSSLAGPSSEMETSVPPGFKVFGAPNVVEDEIDQYLSKQDGKIYRSRDPQLCRHGPLGKCVHCVPLEPFDEDYLNHLEPPVKHMSFHAYIRKLTGGADKGKFVALENISCKIKSGCEGHLPWPNGICTKCQPSAITLNRQKYRHVDNIMFENHTVADRFLDFWRKTGNQHFGYLYGRYTEHKDIPLGIRAEVAAIYEPPQIGTQNSLELLEDPKAEVVDEIAAKLGLRKVGWIFTDLVSEDTRKGTVRYSRNKDTYFLSSEECITAGDFQNKHPNMCRLSPDGHFGSKFVTAVATGGPDNQVHFEGYQVSNQCMALVRDECLLPCKDAPELGYAKESSSEQYVPDVFYKDVDKFGNEITQLARPLPVEYLIIDITTTFPKDPVYTFSISQNPFPIENRDVLGETQDFHSLATYLSQNTSSVFLDTISDFHLLLFLVTNEVMPLQDSISLLLEAVRTRNEELAQTWKRSEQWATIEQLCSTVGGQLPGLHEYGAVGGSTHTATAAMWACQHCTFMNQPGTGHCEMCSLPRT",
# HMGC2_HUMAN
"MGNVPSAVKHCLSYQQLLREHLWIGDSVAGALDPAQTSLLTNLHCFQPDVSGFSVSLAGTVACIHWETSQLSGLPEFVKIVEVGPRDGLQNEKVIVPTDIKIEFINRLSQTGLSVIEVTSFVSSRWVPQMADHTEVMKGIHQYPGVRYPVLTPNLQGFHHAVAAGATEISVFGAASESFSKKNINCSIEESMGKFEEVVKSARHMNIPARGYVSCALGCPYEGSITPQKVTEVSKRLYGMGCYEISLGDTIGVGTPGSMKRMLESVMKEIPPGALAVHCHDTYGQALANILTALQMGINVVDSAVSGLGGCPYAKGASGNVATEDLIYMLNGLGLNTGVNLYKVMEAGDFICKAVNKTTNSKVAQASFNA",
# DJB14_HUMAN
"MEGNRDEAEKCVEIAREALNAGNREKAQRFLQKAEKLYPLPSARALLEIIMKNGSTAGNSPHCRKPSGSGDQSKPNCTKDSTSGSGEGGKGYTKDQVDGVLSINKCKNYYEVLGVTKDAGDEDLKKAYRKLALKFHPDKNHAPGATDAFKKIGNAYAVLSNPEKRKQYDLTGNEEQACNHQNNGRFNFHRGCEADITPEDLFNIFFGGGFPSGSVHSFSNGRAGYSQQHQHRHSGHEREEERGDGGFSVFIQLMPIIVLILVSLLSQLMVSNPPYSLYPRSGTGQTIKMQTENLGVVYYVNKDFKNEYKGMLLQKVEKSVEEDYVTNIRNNCWKERQQKTDMQYAAKVYRDDRLRRKADALSMDNCKELERLTSLYKGG",
# CPT1C_HUMAN
"MAEAHQAVGFRPSLTSDGAEVELSAPVLQEIYLSGLRSWKRHLSRFWNDFLTGVFPASPLSWLFLFSAIQLAWFLQLDPSLGLMEKIKELLPDWGGQHHGLRGVLAAALFASCLWGALIFTLHVALRLLLSYHGWLLEPHGAMSSPTKTWLALVRIFSGRHPMLFSYQRSLPRQPVPSVQDTVRKYLESVRPILSDEDFDWTAVLAQEFLRLQASLLQWYLRLKSWWASNYVSDWWEEFVYLRSRNPLMVNSNYYMMDFLYVTPTPLQAARAGNAVHALLLYRHRLNRQEIPPTLLMGMRPLCSAQYEKIFNTTRIPGVQKDYIRHLHDSQHVAVFHRGRFFRMGTHSRNSLLSPRALEQQFQRILDDPSPACPHEEHLAALTAAPRGTWAQVRTSLKTQAAEALEAVEGAAFFVSLDAEPAGLTREDPAASLDAYAHALLAGRGHDRWFDKSFTLIVFSNGKLGLSVEHSWADCPISGHMWEFTLATECFQLGYSTDGHCKGHPDPTLPQPQRLQWDLPDQIHSSISLALRGAKILSENVDCHVVPFSLFGKSFIRRCHLSSDSFIQIALQLAHFRDRGQFCLTYESAMTRLFLEGRTETVRSCTREACNFVRAMEDKEKTDPQCLALFRVAVDKHQALLKAAMSGQGVDRHLFALYIVSRFLHLQSPFLTQVHSEQWQLSTSQIPVQQMHLFDVHNYPDYVSSGGGFGPADDHGYGVSYIFMGDGMITFHISSKKSSTKTDSHRLGQHIEDALLDVASLFQAGQHFKRRFRGSGKENSRHRCGFLSRQTGASKASMTSTDF",
# STT3B_HUMAN
"MAEPSAPESKHKSSLNSSPWSGLMALGNSRHGHHGPGAQCAHKAAGGAAPPKPAPAGLSGGLSQPAGWQSLLSFTILFLAWLAGFSSRLFAVIRFESIIHEFDPWFNYRSTHHLASHGFYEFLNWFDERAWYPLGRIVGGTVYPGLMITAGLIHWILNTLNITVHIRDVCVFLAPTFSGLTSISTFLLTRELWNQGAGLLAACFIAIVPGYISRSVAGSFDNEGIAIFALQFTYYLWVKSVKTGSVFWTMCCCLSYFYMVSAWGGYVFIINLIPLHVFVLLLMQRYSKRVYIAYSTFYIVGLILSMQIPFVGFQPIRTSEHMAAAGVFALLQAYAFLQYLRDRLTKQEFQTLFFLGVSLAAGAVFLSVIYLTYTGYIAPWSGRFYSLWDTGYAKIHIPIIASVSEHQPTTWVSFFFDLHILVCTFPAGLWFCIKNINDERVFVALYAISAVYFAGVMVRLMLTLTPVVCMLSAIAFSNVFEHYLGDDMKRENPPVEDSSDEDDKRNQGNLYDKAGKVRKHATEQEKTEEGLGPNIKSIVTMLMLMLLMMFAVHCTWVTSNAYSSPSVVLASYNHDGTRNILDDFREAYFWLRQNTDEHARVMSWWDYGYQIAGMANRTTLVDNNTWNNSHIALVGKAMSSNETAAYKIMRTLDVDYVLVIFGGVIGYSGDDINKFLWMVRIAEGEHPKDIRESDYFTPQGEFRVDKAGSPTLLNCLMYKMSYYRFGEMQLDFRTPPGFDRTRNAEIGNKDIKFKHLEEAFTSEHWLVRIYKVKAPDNRETLDHKPRVTNIFPKQKYLSKKTTKRKRGYIKNKLVFKKGKKISKKTV",
# SPP_HUMAN
"MDSALSDPHNGSAEAGGPTNSTTRPPSTPEGIALAYGSLLLMALLPIFFGALRSVRCARGKNASDMPETITSRDAARFPIIASCTLLGLYLFFKIFSQEYINLLLSMYFFVLGILALSHTISPFMNKFFPASFPNRQYQLLFTQGSGENKEEIINYEFDTKDLVCLGLSSIVGVWYLLRKHWIANNLFGLAFSLNGVELLHLNNVSTGCILLGGLFIYDVFWVFGTNVMVTVAKSFEAPIKLVFPQDLLEKGLEANNFAMLGLGDVVIPGIFIALLLRFDISLKKNTHTYFYTSFAAYIFGLGLTIFIMHIFKHAQPALLYLVPACIGFPVLVALAKGEVTEMFSYEESNPKDPAAVTESKEGTEASASKGLEKKEK",
# MRAP_HUMAN
"MANGTNASAPYYSYEYYLDYLDLIPVDEKKLKAHKHSIVIAFWVSLAAFVVLLFLILLYMSWSASPQMRNSPKHHQTCPWSHGLNLHLCIQKCLPCHREPLATSQAQASSVEPGSRTGPDQPLRQESSSTLPLGGFQTHPTLLWELTLNGGPLVRSKPSEPPPGDRTSQLQS",
# ACER1_HUMAN
"MPSIFAYQSSEVDWCESNFQYSELVAEFYNTFSNIPFFIFGPLMMLLMHPYAQKRSRYIYVVWVLFMIIGLFSMYFHMTLSFLGQLLDEIAILWLLGSGYSIWMPRCYFPSFLGGNRSQFIRLVFITTVVSTLLSFLRPTVNAYALNSIALHILYIVCQEYRKTSNKELRHLIEVSVVLWAVALTSWISDRLLCSFWQRIHFFYLHSIWHVLISITFPYGMVTMALVDANYEMPGETLKVRYWPRDSWPVGLPYVEIRGDDKDC",
# RHBL4_HUMAN
"MQRRSRGINTGLILLLSQIFHVGINNIPPVTLATLALNIWFFLNPQKPLYSSCLSVEKCYQQKDWQRLLLSPLHHADDWHLYFNMASMLWKGINLERRLGSRWFAYVITAFSVLTGVVYLLLQFAVAEFMDEPDFKRSCAVGFSGVLFALKVLNNHYCPGGFVNILGFPVPNRFACWVELVAIHLFSPGTSFAGHLAGILVGLMYTQGPLKKIMEACAGGFSSSVGYPGRQYYFNSSGSSGYQDYYPHGRPDHYEEAPRNYDTYTAGLSEEEQLERALQASLWDRGNTRNSPPPYGFHLSPEEMRRQRLHRFDSQ",
# CR3L4_HUMAN
"MDLGIPDLLDAWLEPPEDIFSTGSVLELGLHCPPPEVPVTRLQEQGLQGWKSGGDRGCGLQESEPEDFLKLFIDPNEVYCSEASPGSDSGISEDPCHPDSPPAPRATSSPMLYEVVYEAGALERMQGETGPNVGLISIQLDQWSPAFMVPDSCMVSELPFDAHAHILPRAGTVAPVPCTTLLPCQTLFLTDEEKRLLGQEGVSLPSHLPLTKAEERVLKKVRRKIRNKQSAQDSRRRKKEYIDGLESRVAACSAQNQELQKKVQELERHNISLVAQLRQLQTLIAQTSNKAAQTSTCVLILLFSLALIILPSFSPFQSRPEAGSEDYQPHGVTSRNILTHKDVTENLETQVVESRLREPPGAKDANGSTRTLLEKMGGKPRPSGRIRSVLHADEM",
# ZDHC1_HUMAN
"MYKMNICNKPSNKTAPEKSVWTAPAQPSGPSPELQGQRSRRNGWSWPPHPLQIVAWLLYLFFAVIGFGILVPLLPHHWVPAGYACMGAIFAGHLVVHLTAVSIDPADANVRDKSYAGPLPIFNRSQHAHVIEDLHCNLCNVDVSARSKHCSACNKCVCGFDHHCKWLNNCVGERNYRLFLHSVASALLGVLLLVLVATYVFVEFFVNPMRLRTNRHFEVLKNHTDVWFVFLPAAPVETQAPAILALAALLILLGLLSTALLGHLLCFHIYLMWHKLTTYEYIVQHRPPQEAKGVHRELESCPPKMRPIQEMEFYMRTFRHMRPEPPGQAGPAAVNAKHSRPASPDPTPGRRDCAGPPVQVEWDRKKPLPWRSPLLLLAMWGPQAPPCLCRKRGRGACIKCERLRPRIRRRGLGPPAAAPARRRIPRTPALCTPLALPAPTTRRRQSPWTRFQWRRRAWAAPLWPPRGAGADSPRWRGRRVRPPFS",
# RN139_HUMAN
"MAAVGPPQQQVRMAHQQVWAALEVALRVPCLYIIDAIFNSYPDSSQSRFCIVLQIFLRLFGVFASSIVLILSQRSLFKFYTYSSAFLLAATSVLVNYYASLHIDFYGAYNTSAFGIELLPRKGPSLWMALIVLQLTFGIGYVTLLQIHSIYSQLIILDLLVPVIGLITELPLHIRETLLFTSSLILTLNTVFVLAVKLKWFYYSTRYVYLLVRHMYRIYGLQLLMEDTWKRIRFPDILRVFWLTRVTAQATVLMYILRMANETDSFFISWDDFWDLICNLIISGCDSTLTVLGMSAVISSVAHYLGLGILAFIGSTEEDDRRLGFVAPVLFFILALQTGLSGLRPEERLIRLSRNMCLLLTAVLHFIHGMTDPVLMSLSASHVSSFRRHFPVLFVSACLFILPVLLSYVLWHHYALNTWLFAVTAFCVELCLKVIVSLTVYTLFMIDGYYNVLWEKLDDYVYYVRSTGSIIEFIFGVVMFGNGAYTMMFESGSKIRAFMMCLHAYFNIYLQAKNGWKTFMNRRTAVKKINSLPEIKGSRLQEINDVCAICYHEFTTSARITPCNHYFHALCLRKWLYIQDTCPMCHQKVYIEDDIKDNSNVSNNNGFIPPNETPEEAVREAAAESDRELNEDDSTDCDDDVQRERNGVIQHTGAAAEEFNDDTD",
# ZFN2B_HUMAN
"MEFPDLGAHCSEPSCQRLDFLPLKCDACSGIFCADHVAYAQHHCGSAYQKDIQVPVCPLCNVPVPVARGEPPDRAVGEHIDRDCRSDPAQQKRKIFTNKCERAGCRQREMMKLTCERCSRNFCIKHRHPLDHDCSGEGHPTSRAGLAAISRAQAVASTSTVPSPSQTMPSCTSPSRATTRSPSWTAPPVIALQNGLSEDEALQRALEMSLAETKPQVPSCQEEEDLALAQALSASEAEYQRQQAQSRSSKPSNCSLC",
# APH1B_HUMAN
"MTAAVFFGCAFIAFGPALALYVFTIATEPLRIIFLIAGAFFWLVSLLISSLVWFMARVIIDNKDGPTQKYLLIFGAFVSVYIQEMFRFAYYKLLKKASEGLKSINPGETAPSMRLLAYVSGLGFGIMSGVFSFVNTLSDSLGPGTVGIHGDSPQFFLYSAFMTLVIILLHVFWGIVFFDGCEKKKWGILLIVLLTHLLVSAQTFISSYYGINLASAFIILVLMGTWAFLAAGGSCRSLKLCLLCQDKNFLLYNQRSR",
# GIMA1_HUMAN
"MGGRKMATDEENVYGLEENAQSRQESTRRLILVGRTGAGKSATGNSILGQRRFFSRLGATSVTRACTTGSRRWDKCHVEVVDTPDIFSSQVSKTDPGCEERGHCYLLSAPGPHALLLVTQLGRFTAQDQQAVRQVRDMFGEDVLKWMVIVFTRKEDLAGGSLHDYVSNTENRALRELVAECGGRVCAFDNRATGREQEAQVEQLLGMVEGLVLEHKGAHYSNEVYELAQVLRWAGPEERLRRVAERVAARVQRRPWGAWLSARLWKWLKSPRSWRLGLALLLGGALLFWVLLHRRWSEAVAEVGPD",
# ATLA1_HUMAN
"MAKNRRDRNSWGGFSEKTYEWSSEEEEPVKKAGPVQVLIVKDDHSFELDETALNRILLSEAVRDKEVVAVSVAGAFRKGKSFLMDFMLRYMYNQESVDWVGDYNEPLTGFSWRGGSERETTGIQIWSEIFLINKPDGKKVAVLLMDTQGTFDSQSTLRDSATVFALSTMISSIQVYNLSQNVQEDDLQHLQLFTEYGRLAMEETFLKPFQSLIFLVRDWSFPYEFSYGADGGAKFLEKRLKVSGNQHEELQNVRKHIHSCFTNISCFLLPHPGLKVATNPNFDGKLKEIDDEFIKNLKILIPWLLSPESLDIKEINGNKITCRGLVEYFKAYIKIYQGEELPHPKSMLQATAEANNLAAVATAKDTYNKKMEEICGGDKPFLAPNDLQTKHLQLKEESVKLFRGVKKMGGEEFSRRYLQQLESEIDELYIQYIKHNDSKNIFHAARTPATLFVVIFITYVIAGVTGFIGLDIIASLCNMIMGLTLITLCTWAYIRYSGEYRELGAVIDQVAAALWDQGSTNEALYKLYSAAATHRHLYHQAFPTPKSESTEQSEKKKM",
# RSAD2_HUMAN
"MWVLTPAAFAGKLLSVFRQPLSSLWRSLVPLFCWLRATFWLLATKRRKQQLVLRGPDETKEEEEDPPLPTTPTSVNYHFTRQCNYKCGFCFHTAKTSFVLPLEEAKRGLLLLKEAGMEKINFSGGEPFLQDRGEYLGKLVRFCKVELRLPSVSIVSNGSLIRERWFQNYGEYLDILAISCDSFDEEVNVLIGRGQGKKNHVENLQKLRRWCRDYRVAFKINSVINRFNVEEDMTEQIKALNPVRWKVFQCLLIEGENCGEDALREAERFVIGDEEFERFLERHKEVSCLVPESNQKMKDSYLILDEYMRFLNCRKGRKDPSKSILDVGVEEAIKFSGFDEKMFLKRGGKYIWSKADLKLDW",
# S39A7_HUMAN
"MARGLGAPHWVAVGLLTWATLGLLVAGLGGHDDLHDDLQEDFHGHSHRHSHEDFHHGHSHAHGHGHTHESIWHGHTHDHDHGHSHEDLHHGHSHGYSHESLYHRGHGHDHEHSHGGYGESGAPGIKQDLDAVTLWAYALGATVLISAAPFFVLFLIPVESNSPRHRSLLQILLSFASGGLLGDAFLHLIPHALEPHSHHTLEQPGHGHSHSGQGPILSVGLWVLSGIVAFLVVEKFVRHVKGGHGHSHGHGHAHSHTRGSHGHGRQERSTKEKQSSEEEEKETRGVQKRRGGSTVPKDGPVRPQNAEEEKRGLDLRVSGYLNLAADLAHNFTDGLAIGASFRGGRGLGILTTMTVLLHEVPHEVGDFAILVQSGCSKKQAMRLQLLTAVGALAGTACALLTEGGAVGSEIAGGAGPGWVLPFTAGGFIYVATVSVLPELLREASPLQSLLEVLGLLGGVIMMVLIAHLE",
# PIEZ1_HUMAN
"MEPHVLGAVLYWLLLPCALLAACLLRFSGLSLVYLLFLLLLPWFPGPTRCGLQGHTGRLLRALLGLSLLFLVAHLALQICLHIVPRLDQLLGPSCSRWETLSRHIGVTRLDLKDIPNAIRLVAPDLGILVVSSVCLGICGRLARNTRQSPHPRELDDDERDVDASPTAGLQEAATLAPTRRSRLAARFRVTAHWLLVAAGRVLAVTLLALAGIAHPSALSSVYLLLFLALCTWWACHFPISTRGFSRLCVAVGCFGAGHLICLYCYQMPLAQALLPPAGIWARVLGLKDFVGPTNCSSPHALVLNTGLDWPVYASPGVLLLLCYATASLRKLRAYRPSGQRKEAAKGYEARELELAELDQWPQERESDQHVVPTAPDTEADNCIVHELTGQSSVLRRPVRPKRAEPREASPLHSLGHLIMDQSYVCALIAMMVWSITYHSWLTFVLLLWACLIWTVRSRHQLAMLCSPCILLYGMTLCCLRYVWAMDLRPELPTTLGPVSLRQLGLEHTRYPCLDLGAMLLYTLTFWLLLRQFVKEKLLKWAESPAALTEVTVADTEPTRTQTLLQSLGELVKGVYAKYWIYVCAGMFIVVSFAGRLVVYKIVYMFLFLLCLTLFQVYYSLWRKLLKAFWWLVVAYTMLVLIAVYTFQFQDFPAYWRNLTGFTDEQLGDLGLEQFSVSELFSSILVPGFFLLACILQLHYFHRPFMQLTDMEHVSLPGTRLPRWAHRQDAVSGTPLLREEQQEHQQQQQEEEEEEEDSRDEGLGVATPHQATQVPEGAAKWGLVAERLLELAAGFSDVLSRVQVFLRRLLELHVFKLVALYTVWVALKEVSVMNLLLVVLWAFALPYPRFRPMASCLSTVWTCVIIVCKMLYQLKVVNPQEYSSNCTEPFPNSTNLLPTEISQSLLYRGPVDPANWFGVRKGFPNLGYIQNHLQVLLLLVFEAIVYRRQEHYRRQHQLAPLPAQAVFASGTRQQLDQDLLGCLKYFINFFFYKFGLEICFLMAVNVIGQRMNFLVTLHGCWLVAILTRRHRQAIARLWPNYCLFLALFLLYQYLLCLGMPPALCIDYPWRWSRAVPMNSALIKWLYLPDFFRAPNSTNLISDFLLLLCASQQWQVFSAERTEEWQRMAGVNTDRLEPLRGEPNPVPNFIHCRSYLDMLKVAVFRYLFWLVLVVVFVTGATRISIFGLGYLLACFYLLLFGTALLQRDTRARLVLWDCLILYNVTVIISKNMLSLLACVFVEQMQTGFCWVIQLFSLVCTVKGYYDPKEMMDRDQDCLLPVEEAGIIWDSVCFFFLLLQRRVFLSHYYLHVRADLQATALLASRGFALYNAANLKSIDFHRRIEEKSLAQLKRQMERIRAKQEKHRQGRVDRSRPQDTLGPKDPGLEPGPDSPGGSSPPRRQWWRPWLDHATVIHSGDYFLFESDSEEEEEAVPEDPRPSAQSAFQLAYQAWVTNAQAVLRRRQQEQEQARQEQAGQLPTGGGPSQEVEPAEGPEEAAAGRSHVVQRVLSTAQFLWMLGQALVDELTRWLQEFTRHHGTMSDVLRAERYLLTQELLQGGEVHRGVLDQLYTSQAEATLPGPTEAPNAPSTVSSGLGAEEPLSSMTDDMGSPLSTGYHTRSGSEEAVTDPGEREAGASLYQGLMRTASELLLDRRLRIPELEEAELFAEGQGRALRLLRAVYQCVAAHSELLCYFIIILNHMVTASAGSLVLPVLVFLWAMLSIPRPSKRFWMTAIVFTEIAVVVKYLFQFGFFPWNSHVVLRRYENKPYFPPRILGLEKTDGYIKYDLVQLMALFFHRSQLLCYGLWDHEEDSPSKEHDKSGEEEQGAEEGPGVPAATTEDHIQVEARVGPTDGTPEPQVELRPRDTRRISLRFRRRKKEGPARKGAAAIEAEDREEEEGEEEKEAPTGREKRPSRSGGRVRAAGRRLQGFCLSLAQGTYRPLRRFFHDILHTKYRAATDVYALMFLADVVDFIIIIFGFWAFGKHSAATDITSSLSDDQVPEAFLVMLLIQFSTMVVDRALYLRKTVLGKLAFQVALVLAIHLWMFFILPAVTERMFNQNVVAQLWYFVKCIYFALSAYQIRCGYPTRILGNFLTKKYNHLNLFLFQGFRLVPFLVELRAVMDWVWTDTTLSLSSWMCVEDIYANIFIIKCSRETEKKYPQPKGQKKKKIVKYGMGGLIILFLIAIIWFPLLFMSLVRSVVGVVNQPIDVTVTLKLGGYEPLFTMSAQQPSIIPFTAQAYEELSRQFDPQPLAMQFISQYSPEDIVTAQIEGSSGALWRISPPSRAQMKRELYNGTADITLRFTWNFQRDLAKGGTVEYANEKHMLALAPNSTARRQLASLLEGTSDQSVVIPNLFPKYIRAPNGPEANPVKQLQPNEEADYLGVRIQLRREQGAGATGFLEWWVIELQECRTDCNLLPMVIFSDKVSPPSLGFLAGYGIMGLYVSIVLVIGKFVRGFFSEISHSIMFEELPCVDRILKLCQDIFLVRETRELELEEELYAKLIFLYRSPETMIKWTREKE",
# NICA_HUMAN
"MATAGGGSGADPGSRGLLRLLSFCVLLAGLCRGNSVERKIYIPLNKTAPCVRLLNATHQIGCQSSISGDTGVIHVVEKEEDLQWVLTDGPNPPYMVLLESKHFTRDLMEKLKGRTSRIAGLAVSLTKPSPASGFSPSVQCPNDGFGVYSNSYGPEFAHCREIQWNSLGNGLAYEDFSFPIFLLEDENETKVIKQCYQDHNLSQNGSAPTFPLCAMQLFSHMHAVISTATCMRRSSIQSTFSINPEIVCDPLSDYNVWSMLKPINTTGTLKPDDRVVVAATRLDSRSFFWNVAPGAESAVASFVTQLAAAEALQKAPDVTTLPRNVMFVFFQGETFDYIGSSRMVYDMEKGKFPVQLENVDSFVELGQVALRTSLELWMHTDPVSQKNESVRNQVEDLLATLEKSGAGVPAVILRRPNQSQPLPPSSLQRFLRARNISGVVLADHSGAFHNKYYQSIYDTAENINVSYPEWLSPEEDLNFVTDTAKALADVATVLGRALYELAGGTNFSDTVQADPQTVTRLLYGFLIKANNSWFQSILRQDLRSYLGDGPLQHYIAVSSPTNTTYVVQYALANLTGTVVNLTREQCQDPSKVPSENKDLYEYSWVQGPLHSNETDRLPRCVRSTARLARALSPAFELSQWSSTEYSTWTESRWKDIRARIFLIASKELELITLTVGFGILIFSLIVTYCINAKADVLFIAPREPGAVSY",
# UBXN4_HUMAN
"MLWFQGAIPAAIATAKRSGAVFVVFVAGDDEQSTQMAASWEDDKVTEASSNSFVAIKIDTKSEACLQFSQIYPVVCVPSSFFIGDSGIPLEVIAGSVSADELVTRIHKVRQMHLLKSETSVANGSQSESSVSTPSASFEPNNTCENSQSRNAELCEIPPTSDTKSDTATGGESAGHATSSQEPSGCSDQRPAEDLNIRVERLTKKLEERREEKRKEEEQREIKKEIERRKTGKEMLDYKRKQEEELTKRMLEERNREKAEDRAARERIKQQIALDRAERAARFAKTKEEVEAAKAAALLAKQAEMEVKRESYARERSTVARIQFRLPDGSSFTNQFPSDAPLEEARQFAAQTVGNTYGNFSLATMFPRREFTKEDYKKKLLDLELAPSASVVLLPAGRPTASIVHSSSGDIWTLLGTVLYPFLAIWRLISNFLFSNPPPTQTSVRVTSSEPPNPASSSKSEKREPVRKRVLEKRGDDFKKEGKIYRLRTQDDGEDENNTWNGNSTQQM",
# EDEM1_HUMAN
"MQWRALVLGLVLLRLGLHGVLWLVFGLGPSMGFYQRFPLSFGFQRLRSPDGPASPTSGPVGRPGGVSGPSWLQPPGTGAAQSPRKAPRRPGPGMCGPANWGYVLGGRGRGPDEYEKRYSGAFPPQLRAQMRDLARGMFVFGYDNYMAHAFPQDELNPIHCRGRGPDRGDPSNLNINDVLGNYSLTLVDALDTLAIMGNSSEFQKAVKLVINTVSFDKDSTVQVFEATIRVLGSLLSAHRIITDSKQPFGDMTIKDYDNELLYMAHDLAVRLLPAFENTKTGIPYPRVNLKTGVPPDTNNETCTAGAGSLLVEFGILSRLLGDSTFEWVARRAVKALWNLRSNDTGLLGNVVNIQTGHWVGKQSGLGAGLDSFYEYLLKSYILFGEKEDLEMFNAAYQSIQNYLRRGREACNEGEGDPPLYVNVNMFSGQLMNTWIDSLQAFFPGLQVLIGDVEDAICLHAFYYAIWKRYGALPERYNWQLQAPDVLFYPLRPELVESTYLLYQATKNPFYLHVGMDILQSLEKYTKVKCGYATLHHVIDKSTEDRMESFFLSETCKYLYLLFDEDNPVHKSGTRYMFTTEGHIVSVDEHLRELPWKEFFSEEGGQDQGGKSVHRPKPHELKVINSSSNCNRVPDERRYSLPLKSIYMRQIDQMVGLI",
# SORL_HUMAN
"MATRSSRRESRLPFLFTLVALLPPGALCEVWTQRLHGGSAPLPQDRGFLVVQGDPRELRLWARGDARGASRADEKPLRRKRSAALQPEPIKVYGQVSLNDSHNQMVVHWAGEKSNVIVALARDSLALARPKSSDVYVSYDYGKSFKKISDKLNFGLGNRSEAVIAQFYHSPADNKRYIFADAYAQYLWITFDFCNTLQGFSIPFRAADLLLHSKASNLLLGFDRSHPNKQLWKSDDFGQTWIMIQEHVKSFSWGIDPYDKPNTIYIERHEPSGYSTVFRSTDFFQSRENQEVILEEVRDFQLRDKYMFATKVVHLLGSEQQSSVQLWVSFGRKPMRAAQFVTRHPINEYYIADASEDQVFVCVSHSNNRTNLYISEAEGLKFSLSLENVLYYSPGGAGSDTLVRYFANEPFADFHRVEGLQGVYIATLINGSMNEENMRSVITFDKGGTWEFLQAPAFTGYGEKINCELSQGCSLHLAQRLSQLLNLQLRRMPILSKESAPGLIIATGSVGKNLASKTNVYISSSAGARWREALPGPHYYTWGDHGGIITAIAQGMETNELKYSTNEGETWKTFIFSEKPVFVYGLLTEPGEKSTVFTIFGSNKENVHSWLILQVNATDALGVPCTENDYKLWSPSDERGNECLLGHKTVFKRRTPHATCFNGEDFDRPVVVSNCSCTREDYECDFGFKMSEDLSLEVCVPDPEFSGKSYSPPVPCPVGSTYRRTRGYRKISGDTCSGGDVEARLEGELVPCPLAEENEFILYAVRKSIYRYDLASGATEQLPLTGLRAAVALDFDYEHNCLYWSDLALDVIQRLCLNGSTGQEVIINSGLETVEALAFEPLSQLLYWVDAGFKKIEVANPDGDFRLTIVNSSVLDRPRALVLVPQEGVMFWTDWGDLKPGIYRSNMDGSAAYHLVSEDVKWPNGISVDDQWIYWTDAYLECIERITFSGQQRSVILDNLPHPYAIAVFKNEIYWDDWSQLSIFRASKYSGSQMEILANQLTGLMDMKIFYKGKNTGSNACVPRPCSLLCLPKANNSRSCRCPEDVSSSVLPSGDLMCDCPQGYQLKNNTCVKQENTCLRNQYRCSNGNCINSIWWCDFDNDCGDMSDERNCPTTICDLDTQFRCQESGTCIPLSYKCDLEDDCGDNSDESHCEMHQCRSDEYNCSSGMCIRSSWVCDGDNDCRDWSDEANCTAIYHTCEASNFQCRNGHCIPQRWACDGDTDCQDGSDEDPVNCEKKCNGFRCPNGTCIPSSKHCDGLRDCSDGSDEQHCEPLCTHFMDFVCKNRQQCLFHSMVCDGIIQCRDGSDEDAAFAGCSQDPEFHKVCDEFGFQCQNGVCISLIWKCDGMDDCGDYSDEANCENPTEAPNCSRYFQFRCENGHCIPNRWKCDRENDCGDWSDEKDCGDSHILPFSTPGPSTCLPNYYRCSSGTCVMDTWVCDGYRDCADGSDEEACPLLANVTAASTPTQLGRCDRFEFECHQPKTCIPNWKRCDGHQDCQDGRDEANCPTHSTLTCMSREFQCEDGEACIVLSERCDGFLDCSDESDEKACSDELTVYKVQNLQWTADFSGDVTLTWMRPKKMPSASCVYNVYYRVVGESIWKTLETHSNKTNTVLKVLKPDTTYQVKVQVQCLSKAHNTNDFVTLRTPEGLPDAPRNLQLSLPREAEGVIVGHWAPPIHTHGLIREYIVEYSRSGSKMWASQRAASNFTEIKNLLVNTLYTVRVAAVTSRGIGNWSDSKSITTIKGKVIPPPDIHIDSYGENYLSFTLTMESDIKVNGYVVNLFWAFDTHKQERRTLNFRGSILSHKVGNLTAHTSYEISAWAKTDLGDSPLAFEHVMTRGVRPPAPSLKAKAINQTAVECTWTGPRNVVYGIFYATSFLDLYRNPKSLTTSLHNKTVIVSKDEQYLFLVRVVVPYQGPSSDYVVVKMIPDSRLPPRHLHVVHTGKTSVVIKWESPYDSPDQDLLYAVAVKDLIRKTDRSYKVKSRNSTVEYTLNKLEPGGKYHIIVQLGNMSKDSSIKITTVSLSAPDALKIITENDHVLLFWKSLALKEKHFNESRGYEIHMFDSAMNITAYLGNTTDNFFKISNLKMGHNYTFTVQARCLFGNQICGEPAILLYDELGSGADASATQAARSTDVAAVVVPILFLILLSLGVGFAILYTKHRRLQSSFTAFANSHYSSRLGSAIFSSGDDLGEDDEDAPMITGFSDDVPMVIA",
# UFD1_HUMAN
"MFSFNMFDHPIPRVFQNRFSTQYRCFSVSMLAGPNDRSDVEKGGKIIMPPSALDQLSRLNITYPMLFKLTNKNSDRMTHCGVLEFVADEGICYLPHWMMQNLLLEEGGLVQVESVNLQVATYSKFQPQSPDFLDITNPKAVLENALRNFACLTTGDVIAINYNEKIYELRVMETKPDKAVSIIECDMNVDFDAPLGYKEPERQVQHEESTEGEADHSGYAGELGFRAFSGSGNRLDGKKKGVEPSPSPIKPGDIKRGIPNYEFKLGKITFIRNSRPLVKKVEEDEAGGRFVAFSGEGQSLRKKGRKP",
# CDS1_HUMAN
"MLELRHRGSCPGPREAVSPPHREGEAAGGDHETESTSDKETDIDDRYGDLDSRTDSDIPEIPPSSDRTPEILKKALSGLSSRWKNWWIRGILTLTMISLFFLIIYMGSFMLMLLVLGIQVKCFHEIITIGYRVYHSYDLPWFRTLSWYFLLCVNYFFYGETVADYFATFVQREEQLQFLIRYHRFISFALYLAGFCMFVLSLVKKHYRLQFYMFAWTHVTLLITVTQSHLVIQNLFEGMIWFLVPISSVICNDITAYLFGFFFGRTPLIKLSPKKTWEGFIGGFFSTVVFGFIAAYVLSKYQYFVCPVEYRSDVNSFVTECEPSELFQLQTYSLPPFLKAVLRQERVSLYPFQIHSIALSTFASLIGPFGGFFASGFKRAFKIKDFANTIPGHGGIMDRFDCQYLMATFVHVYITSFIRGPNPSKVLQQLLVLQPEQQLNIYKTLKTHLIEKGILQPTLKV",
# EXT2_HUMAN
"MCASVKYNIRGPALIPRMKTKHRIYYITLFSIVLLGLIATGMFQFWPHSIESSNDWNVEKRSIRDVPVVRLPADSPIPERGDLSCRMHTCFDVYRCGFNPKNKIKVYIYALKKYVDDFGVSVSNTISREYNELLMAISDSDYYTDDINRACLFVPSIDVLNQNTLRIKETAQAMAQLSRWDRGTNHLLFNMLPGGPPDYNTALDVPRDRALLAGGGFSTWTYRQGYDVSIPVYSPLSAEVDLPEKGPGPRQYFLLSSQVGLHPEYREDLEALQVKHGESVLVLDKCTNLSEGVLSVRKRCHKHQVFDYPQVLQEATFCVVLRGARLGQAVLSDVLQAGCVPVVIADSYILPFSEVLDWKRASVVVPEEKMSDVYSILQSIPQRQIEEMQRQARWFWEAYFQSIKAIALATLQIINDRIYPYAAISYEEWNDPPAVKWGSVSNPLFLPLIPPQSQGFTAIVLTYDRVESLFRVITEVSKVPSLSKLLVVWNNQNKNPPEDSLWPKIRVPLKVVRTAENKLSNRFFPYDEIETEAVLAIDDDIIMLTSDELQFGYEVWREFPDRLVGYPGRLHLWDHEMNKWKYESEWTNEVSMVLTGAAFYHKYFNYLYTYKMPGDIKNWVDAHMNCEDIAMNFLVANVTGKAVIKVTPRKKFKCPECTAIDGLSLDQTHMVERSECINKFASVFGTMPLKVVEHRADPVLYKDDFPEKLKSFPNIGSL",
# AT2A3_HUMAN
"MEAAHLLPAADVLRHFSVTAEGGLSPAQVTGARERYGPNELPSEEGKSLWELVLEQFEDLLVRILLLAALVSFVLAWFEEGEETTTAFVEPLVIMLILVANAIVGVWQERNAESAIEALKEYEPEMGKVIRSDRKGVQRIRARDIVPGDIVEVAVGDKVPADLRLIEIKSTTLRVDQSILTGESVSVTKHTEAIPDPRAVNQDKKNMLFSGTNITSGKAVGVAVATGLHTELGKIRSQMAAVEPERTPLQRKLDEFGRQLSHAISVICVAVWVINIGHFADPAHGGSWLRGAVYYFKIAVALAVAAIPEGLPAVITTCLALGTRRMARKNAIVRSLPSVETLGCTSVICSDKTGTLTTNQMSVCRMFVVAEADAGSCLLHEFTISGTTYTPEGEVRQGDQPVRCGQFDGLVELATICALCNDSALDYNEAKGVYEKVGEATETALTCLVEKMNVFDTDLQALSRVERAGACNTVIKQLMRKEFTLEFSRDRKSMSVYCTPTRPHPTGQGSKMFVKGAPESVIERCSSVRVGSRTAPLTPTSREQILAKIRDWGSGSDTLRCLALATRDAPPRKEDMELDDCSKFVQYETDLTFVGCVGMLDPPRPEVAACITRCYQAGIRVVMITGDNKGTAVAICRRLGIFGDTEDVAGKAYTGREFDDLSPEQQRQACRTARCFARVEPAHKSRIVENLQSFNEITAMTGDGVNDAPALKKAEIGIAMGSGTAVAKSAAEMVLSDDNFASIVAAVEEGRAIYSNMKQFIRYLISSNVGEVVCIFLTAILGLPEALIPVQLLWVNLVTDGLPATALGFNPPDLDIMEKLPRSPREALISGWLFFRYLAIGVYVGLATVAAATWWFVYDAEGPHINFYQLRNFLKCSEDNPLFAGIDCEVFESRFPTTMALSVLVTIEMCNALNSVSENQSLLRMPPWMNPWLLVAVAMSMALHFLILLVPPLPLIFQVTPLSGRQWVVVLQISLPVILLDEALKYLSRNHMHEEMSQK",
# HMR1_HUMAN
"MGELMAFLLPLIIVLMVKHSDSRTHSLRYFRLGVSDPIHGVPEFISVGYVDSHPITTYDSVTRQKEPRAPWMAENLAPDHWERYTQLLRGWQQMFKVELKRLQRHYNHSGSHTYQRMIGCELLEDGSTTGFLQYAYDGQDFLIFNKDTLSWLAVDNVAHTIKQAWEANQHELLYQKNWLEEECIAWLKRFLEYGKDTLQRTEPPLVRVNRKETFPGVTALFCKAHGFYPPEIYMTWMKNGEEIVQEIDYGDILPSGDGTYQAWASIELDPQSSNLYSCHVEHCGVHMVLQVPQESETIPLVMKAVSGSIVLVIVLAGVGVLVWRRRPREQNGAIYLPTPDR",
# FATE1_HUMAN
"MAGGPPNTKAEMEMSLAEELNHGRQGENQEHLVIAEMMELGSRSRGASQKKQKLEQKAAGSASAKRVWNMTATRPKKMGSQLPKPRMLRESGHGDAHLQEYAGNFQGIRFHYDRNPGTDAVAQTSLEEFNVLEMEVMRRQLYAVNRRLRALEEQGATWRHRETLIIAVLVSASIANLWLWMNQ",
# YIPF5_HUMAN
"MSGFENLNTDFYQTSYSIDDQSQQSYDYGGSGGPYSKQYAGYDYSQQGRFVPPDMMQPQQPYTGQIYQPTQAYTPASPQPFYGNNFEDEPPLLEELGINFDHIWQKTLTVLHPLKVADGSIMNETDLAGPMVFCLAFGATLLLAGKIQFGYVYGISAIGCLGMFCLLNLMSMTGVSFGCVASVLGYCLLPMILLSSFAVIFSLQGMVGIILTAGIIGWCSFSASKIFISALAMEGQQLLVAYPCALLYGVFALISVF",
# S35B4_HUMAN
"MRPALAVGLVFAGCCSNVIFLELLARKHPGCGNIVTFAQFLFIAVEGFLFEADLGRKPPAIPIRYYAIMVTMFFTVSVVNNYALNLNIAMPLHMIFRSGSLIANMILGIIILKKRYSIFKYTSIALVSVGIFICTFMSAKQVTSQSSLSENDGFQAFVWWLLGIGALTFALLMSARMGIFQETLYKRFGKHSKEALFYNHALPLPGFVFLASDIYDHAVLFNKSELYEIPVIGVTLPIMWFYLLMNIITQYVCIRGVFILTTECASLTVTLVVTLRKFVSLIFSILYFQNPFTLWHWLGTLFVFIGTLMYTEVWNNLGTTKSEPQKDSKKN",
# TM203_HUMAN
"MLFSLRELVQWLGFATFEIFVHLLALLVFSVLLALRVDGLVPGLSWWNVFVPFFAADGLSTYFTTIVSVRLFQDGEKRLAVLRLFWVLTVLSLKFVFEMLLCQKLAEQTRELWFGLITSPLFILLQLLMIRACRVN",
# CCD47_HUMAN
"MKAFHTFCVVLLVFGSVSEAKFDDFEDEEDIVEYDDNDFAEFEDVMEDSVTESPQRVIITEDDEDETTVELEGQDENQEGDFEDADTQEGDTESEPYDDEEFEGYEDKPDTSSSKNKDPITIVDVPAHLQNSWESYYLEILMVTGLLAYIMNYIIGKNKNSRLAQAWFNTHRELLESNFTLVGDDGTNKEATSTGKLNQENEHIYNLWCSGRVCCEGMLIQLRFLKRQDLLNVLARMMRPVSDQVQIKVTMNDEDMDTYVFAVGTRKALVRLQKEMQDLSEFCSDKPKSGAKYGLPDSLAILSEMGEVTDGMMDTKMVHFLTHYADKIESVHFSDQFSGPKIMQEEGQPLKLPDTKRTLLFTFNVPGSGNTYPKDMEALLPLMNMVIYSIDKAKKFRLNREGKQKADKNRARVEENFLKLTHVQRQEAAQSRREEKKRAEKERIMNEEDPEKQRRLEEAALRREQKKLEKKQMKMKQIKVKAM",
# LRC59_HUMAN
"MTKAGSKGGNLRDKLDGNELDLSLSDLNEVPVKELAALPKATILDLSCNKLTTLPSDFCGLTHLVKLDLSKNKLQQLPADFGRLVNLQHLDLLNNKLVTLPVSFAQLKNLKWLDLKDNPLDPVLAKVAGDCLDEKQCKQCANKVLQHMKAVQADQERERQRRLEVEREAEKKREAKQRAKEAQERELRKREKAEEKERRRKEYDALKAAKREQEKKPKKEANQAPKSKSGSRPRKPPPRKHTRSWAVLKLLLLLLLFGVAGGLVACRVTELQQQPLCTSVNTIYDNAVQGLRRHEILQWVLQTDSQQ",
# CR3L1_HUMAN
"MDAVLEPFPADRLFPGSSFLDLGDLNESDFLNNAHFPEHLDHFTENMEDFSNDLFSSFFDDPVLDEKSPLLDMELDSPTPGIQAEHSYSLSGDSAPQSPLVPIKMEDTTQDAEHGAWALGHKLCSIMVKQEQSPELPVDPLAAPSAMAAAAAMATTPLLGLSPLSRLPIPHQAPGEMTQLPVIKAEPLEVNQFLKVTPEDLVQMPPTPPSSHGSDSDGSQSPRSLPPSSPVRPMARSSTAISTSPLLTAPHKLQGTSGPLLLTEEEKRTLIAEGYPIPTKLPLTKAEEKALKRVRRKIKNKISAQESRRKKKEYVECLEKKVETFTSENNELWKKVETLENANRTLLQQLQKLQTLVTNKISRPYKMAATQTGTCLMVAALCFVLVLGSLVPCLPEFSSGSQTVKEDPLAADGVYTASQMPSRSLLFYDDGAGLWEDGRSTLLPMEPPDGWEINPGGPAEQRPRDHLQHDHLDSTHETTKYLSEAWPKDGGNGTSPDFSHSKEWFHDRDLGPNTTIKLS",
# APH1A_HUMAN
"MGAAVFFGCTFVAFGPAFALFLITVAGDPLRVIILVAGAFFWLVSLLLASVVWFILVHVTDRSDARLQYGLLIFGAAVSVLLQEVFRFAYYKLLKKADEGLASLSEDGRSPISIRQMAYVSGLSFGIISGVFSVINILADALGPGVVGIHGDSPYYFLTSAFLTAAIILLHTFWGVVFFDACERRRYWALGLVVGSHLLTSGLTFLNPWYEASLLPIYAVTVSMGLWAFITAGGSLRSIQRSLLCRRQEDSRVMVYSALRIPPED",
# SARAF_HUMAN
"MAAACGPGAAGYCLLLGLHLFLLTAGPALGWNDPDRMLLRDVKALTLHYDRYTTSRRLDPIPQLKCVGGTAGCDSYTPKVIQCQNKGWDGYDVQWECKTDLDIAYKFGKTVVSCEGYESSEDQYVLRGSCGLEYNLDYTELGLQKLKESGKQHGFASFSDYYYKWSSADSCNMSGLITIVVLLGIAFVVYKLFLSDGQYSPPPYSEYPPFSHRYQRFTNSAGPPPPGFKSEFTGPQNTGHGATSGFGSAFTGQQGYENSGPGFWTGLGTGGILGYLFGSNRAATPFSDSWYYPSYPPSYPGTWNRAYSPLHGGSGSYSVCSNSDTKTRTASGYGGTRRR",
# TBC20_HUMAN
"MALRSAQGDGPTSGHWDGGAEKADFNAKRKKKVAEIHQALNSDPTDVAALRRMAISEGGLLTDEIRRKVWPKLLNVNANDPPPISGKNLRQMSKDYQQVLLDVRRSLRRFPPGMPEEQREGLQEELIDIILLILERNPQLHYYQGYHDIVVTFLLVVGERLATSLVEKLSTHHLRDFMDPTMDNTKHILNYLMPIIDQVNPELHDFMQSAEVGTIFALSWLITWFGHVLSDFRHVVRLYDFFLACHPLMPIYFAAVIVLYREQEVLDCDCDMASVHHLLSQIPQDLPYETLISRAGDLFVQFPPSELAREAAAQQQAERTAASTFKDFELASAQQRPDMVLRQRFRGLLRPEDRTKDVLTKPRTNRFVKLAVMGLTVALGAAALAVVKSALEWAPKFQLQLFP",
# FAF2_HUMAN
"MAAPEERDLTQEQTEKLLQFQDLTGIESMDQCRHTLEQHNWNIEAAVQDRLNEQEGVPSVFNPPPSRPLQVNTADHRIYSYVVSRPQPRGLLGWGYYLIMLPFRFTYYTILDIFRFALRFIRPDPRSRVTDPVGDIVSFMHSFEEKYGRAHPVFYQGTYSQALNDAKRELRFLLVYLHGDDHQDSDEFCRNTLCAPEVISLINTRMLFWACSTNKPEGYRVSQALRENTYPFLAMIMLKDRRMTVVGRLEGLIQPDDLINQLTFIMDANQTYLVSERLEREERNQTQVLRQQQDEAYLASLRADQEKERKKREERERKRRKEEEVQQQKLAEERRRQNLQEEKERKLECLPPEPSPDDPESVKIIFKLPNDSRVERRFHFSQSLTVIHDFLFSLKESPEKFQIEANFPRRVLPCIPSEEWPNPPTLQEAGLSHTEVLFVQDLTDE",
# RN183_HUMAN
"MAEQQGRELEAECPVCWNPFNNTFHTPKMLDCCHSFCVECLAHLSLVTPARRRLLCPLCRQPTVLASGQPVTDLPTDTAMLALLRLEPHHVILEGHQLCLKDQPKSRYFLRQPQVYTLDLGPQPGGQTGPPPDTASATVSTPILIPSHHSLRECFRNPQFRIFAYLMAVILSVTLLLIFSIFWTKQFLWGVG",
# CERS2_HUMAN
"MLQTLYDYFWWERLWLPVNLTWADLEDRDGRVYAKASDLYITLPLALLFLIVRYFFELYVATPLAALLNIKEKTRLRAPPNATLEHFYLTSGKQPKQVEVELLSRQSGLSGRQVERWFRRRRNQDRPSLLKKFREASWRFTFYLIAFIAGMAVIVDKPWFYDMKKVWEGYPIQSTIPSQYWYYMIELSFYWSLLFSIASDVKRKDFKEQIIHHVATIILISFSWFANYIRAGTLIMALHDSSDYLLESAKMFNYAGWKNTCNNIFIVFAIVFIITRLVILPFWILHCTLVYPLELYPAFFGYYFFNSMMGVLQLLHIFWAYLILRMAHKFITGKLVEDERSDREETESSEGEEAAAGGGAKSRPLANGHPILNNNHRKND",
# MRAP2_HUMAN
"MSAQRLISNRTSQQSASNSDYTWEYEYYEIGPVSFEGLKAHKYSIVIGFWVGLAVFVIFMFFVLTLLTKTGAPHQDNAESSEKRFRMNSFVSDFGRPLEPDKVFSRQGNEESRSLFHCYINEVERLDRAKACHQTTALDSDVQLQEAIRSSGQPEEELNRLMKFDIPNFVNTDQNYFGEDDLLISEPPIVLETKPLSQTSHKDLD",
# VMP1_HUMAN
"MAENGKNCDQRRVAMNKEHHNGNFTDPSSVNEKKRREREERQNIVLWRQPLITLQYFSLEILVILKEWTSKLWHRQSIVVSFLLLLAVLIATYYVEGVHQQYVQRIEKQFLLYAYWIGLGILSSVGLGTGLHTFLLYLGPHIASVTLAAYECNSVNFPEPPYPDQIICPDEEGTEGTISLWSIISKVRIEACMWGIGTAIGELPPYFMARAARLSGAEPDDEEYQEFEEMLEHAESAQDFASRAKLAVQKLVQKVGFFGILACASIPNPLFDLAGITCGHFLVPFWTFFGATLIGKAIIKMHIQKIFVIITFSKHIVEQMVAFIGAVPGIGPSLQKPFQEYLEAQRQKLHHKSEMGTPQGENWLSWMFEKLVVVMVCYFILSIINSMAQSYAKRIQQRLNSEEKTK",
# RN185_HUMAN
"MASKGPSASASPENSSAGGPSGSSNGAGESGGQDSTFECNICLDTAKDAVISLCGHLFCWPCLHQWLETRPNRQVCPVCKAGISRDKVIPLYGRGSTGQQDPREKTPPRPQGQRPEPENRGGFQGFGFGDGGFQMSFGIGAFPFGIFATAFNINDGRPPPAVPGTPQYVDEQFLSRLFLFVALVIMFWLLIA",
# ZDH12_HUMAN
"MAPWALLSPGVLVRTGHTVLTWGITLVLFLHDTELRQWEEQGELLLPLTFLLLVLGSLLLYLAVSLMDPGYVNVQPQPQEELKEEQTAMVPPAIPLRRCRYCLVLQPLRARHCRECRRCVRRYDHHCPWMENCVGERNHPLFVVYLALQLVVLLWGLYLAWSGLRFFQPWGQWLRSSGLLFATFLLLSLFSLVASLLLVSHLYLVASNTTTWEFISSHRIAYLRQRPSNPFDRGLTRNLAHFFCGWPSGSWETLWAEEEEEGSSPAV",
# S39AD_HUMAN
"MPGCPCPGCGMAGPRLLFLTALALELLERAGGSQPALRSRGTATACRLDNKESESWGALLSGERLDTWICSLLGSLMVGLSGVFPLLVIPLEMGTMLRSEAGAWRLKQLLSFALGGLLGNVFLHLLPEAWAYTCSASPGGEGQSLQQQQQLGLWVIAGILTFLALEKMFLDSKEEGTSQAPNKDPTAAAAALNGGHCLAQPAAEPGLGAVVRSIKVSGYLNLLANTIDNFTHGLAVAASFLVSKKIGLLTTMAILLHEIPHEVGDFAILLRAGFDRWSAAKLQLSTALGGLLGAGFAICTQSPKGVVGCSPAAEETAAWVLPFTSGGFLYIALVNVLPDLLEEEDPWRSLQQLLLLCAGIVVMVLFSLFVD",
# ERO1A_HUMAN
"MGRGWGFLFGLLGAVWLLSSGHGEEQPPETAAQRCFCQVSGYLDDCTCDVETIDRFNNYRLFPRLQKLLESDYFRYYKVNLKRPCPFWNDISQCGRRDCAVKPCQSDEVPDGIKSASYKYSEEANNLIEECEQAERLGAVDESLSEETQKAVLQWTKHDDSSDNFCEADDIQSPEAEYVDLLLNPERYTGYKGPDAWKIWNVIYEENCFKPQTIKRPLNPLASGQGTSEENTFYSWLEGLCVEKRAFYRLISGLHASINVHLSARYLLQETWLEKKWGHNITEFQQRFDGILTEGEGPRRLKNLYFLYLIELRALSKVLPFFERPDFQLFTGNKIQDEENKMLLLEILHEIKSFPLHFDENSFFAGDKKEAHKLKEDFRLHFRNISRIMDCVGCFKCRLWGKLQTQGLGTALKILFSEKLIANMPESGPSYEFHLTRQEIVSLFNAFGRISTSVKELENFRNLLQNIH",
# REEP6_HUMAN
"MDGLRQRVEHFLEQRNLVTEVLGALEAKTGVEKRYLAAGAVTLLSLYLLFGYGASLLCNLIGFVYPAYASIKAIESPSKDDDTVWLTYWVVYALFGLAEFFSDLLLSWFPFYYVGKCAFLLFCMAPRPWNGALMLYQRVVRPLFLRHHGAVDRIMNDLSGRALDAAAGITRNVLQVLARSRAGITPVAVAGPSTPLEADLKPSQTPQPKDK",
# DDRGK_HUMAN
"MVAPVWYLVAAALLVGFILFLTRSRGRAASAGQEPLHNEELAGAGRVAQPGPLEPEEPRAGGRPRRRRDLGSRLQAQRRAQRVAWAEADENEEEAVILAQEEEGVEKPAETHLSGKIGAKKLRKLEEKQARKAQREAEEAEREERKRLESQREAEWKKEEERLRLEEEQKEEEERKAREEQAQREHEEYLKLKEAFVVEEEGVGETMTEEQSQSFLTEFINYIKQSKVVLLEDLASQVGLRTQDTINRIQDLLAEGTITGVIDDRGKFIYITPEELAAVANFIRQRGRVSIAELAQASNSLIAWGRESPAQAPA",
# DNJC1_HUMAN
"MTAPCSQPAQLPGRRQLGLVPFPPPPPRTPLLWLLLLLLAAVAPARGWESGDLELFDLVEEVQLNFYQFLGVQQDASSADIRKAYRKLSLTLHPDKNKDENAETQFRQLVAIYEVLKDDERRQRYDDILINGLPDWRQPVFYYRRVRKMSNAELALLLFIILTVGHYAVVWSIYLEKQLDELLSRKKREKKKKTGSKSVDVSKLGASEKNERLLMKPQWHDLLPCKLGIWFCLTLKALPHLIQDAGQFYAKYKETRLKEKEDALTRTELETLQKQKKVKKPKPEFPVYTPLETTYIQSYDHGTSIEEIEEQMDDWLENRNRTQKKQAPEWTEEDLSQLTRSMVKFPGGTPGRWEKIAHELGRSVTDVTTKAKQLKDSVTCSPGMVRLSELKSTVQNSRPIKTATTLPDDMITQREDAEGVAAEEEQEGDSGEQETGATDARPRRRKPARLLEATAKPEPEEKSRAKRQKDFDIAEQNESSDEESLRKERARSAEEPWTQNQQKLLELALQQYPRGSSDRWDKIARCVPSKSKEDCIARYKLLVELVQKKKQAKS",
# SAMD8_HUMAN
"MAGPNQLCIRRWTTKHVAVWLKDEGFFEYVDILCNKHRLDGITLLTLTEYDLRSPPLEIKVLGDIKRLMLSVRKLQKIHIDVLEEMGYNSDSPMGSMTPFISALQSTDWLCNGELSHDCDGPITDLNSDQYQYMNGKNKHSVRRLDPEYWKTILSCIYVFIVFGFTSFIMVIVHERVPDMQTYPPLPDIFLDSVPRIPWAFAMTEVCGMILCYIWLLVLLLHKHRSILLRRLCSLMGTVFLLRCFTMFVTSLSVPGQHLQCTGKIYGSVWEKLHRAFAIWSGFGMTLTGVHTCGDYMFSGHTVVLTMLNFFVTEYTPRSWNFLHTLSWVLNLFGIFFILAAHEHYSIDVFIAFYITTRLFLYYHTLANTRAYQQSRRARIWFPMFSFFECNVNGTVPNEYCWPFSKPAIMKRLIG",
# DGAT2_HUMAN
"MKTLIAAYSGVLRGERQAEADRSQRSHGGPALSREGSGRWGTGSSILSALQDLFSVTWLNRSKVEKQLQVISVLQWVLSFLVLGVACSAILMYIFCTDCWLIAVLYFTWLVFDWNTPKKGGRRSQWVRNWAVWRYFRDYFPIQLVKTHNLLTTRNYIFGYHPHGIMGLGAFCNFSTEATEVSKKFPGIRPYLATLAGNFRMPVLREYLMSGGICPVSRDTIDYLLSKNGSGNAIIIVVGGAAESLSSMPGKNAVTLRNRKGFVKLALRHGADLVPIYSFGENEVYKQVIFEEGSWGRWVQKKFQKYIGFAPCIFHGRGLFSSDTWGLVPYSKPITTVVGEPITIPKLEHPTQQDIDLYHTMYMEALVKLFDKHKTKFGLPETEVLEVN",
# SIM14_HUMAN
"MAEGGFDPCECVCSHEHAMRRLINLLRQSQSYCTDTECLQELPGPSGDNGISVTMILVAWMVIALILFLLRPPNLRGSSLPGKPTSPHNGQDPPAPPVD",
# PANX1_HUMAN
"MAIAQLATEYVFSDFLLKEPTEPKFKGLRLELAVDKMVTCIAVGLPLLLISLAFAQEISIGTQISCFSPSSFSWRQAAFVDSYCWAAVQQKNSLQSESGNLPLWLHKFFPYILLLFAILLYLPPLFWRFAAAPHICSDLKFIMEELDKVYNRAIKAAKSARDLDMRDGACSVPGVTENLGQSLWEVSESHFKYPIVEQYLKTKKNSNNLIIKYISCRLLTLIIILLACIYLGYYFSLSSLSDEFVCSIKSGILRNDSTVPDQFQCKLIAVGIFQLLSVINLVVYVLLAPVVVYTLFVPFRQKTDVLKVYEILPTFDVLHFKSEGYNDLSLYNLFLEENISEVKSYKCLKVLENIKSSGQGIDPMLLLTNLGMIKMDVVDGKTPMSAEMREEQGNQTAELQGMNIDSETKANNGEKNARQRLLDSSC",
# CLCC1_HUMAN
"MLCSLLLCECLLLVAGYAHDDDWIDPTDMLNYDAASGTMRKSQAKYGISGEKDVSPDLSCADEISECYHKLDSLTYKIDECEKKKREDYESQSNPVFRRYLNKILIEAGKLGLPDENKGDMHYDAEIILKRETLLEIQKFLNGEDWKPGALDDALSDILINFKFHDFETWKWRFEDSFGVDPYNVLMVLLCLLCIVVLVATELWTYVRWYTQLRRVLIISFLFSLGWNWMYLYKLAFAQHQAEVAKMEPLNNVCAKKMDWTGSIWEWFRSSWTYKDDPCQKYYELLLVNPIWLVPPTKALAVTFTTFVTEPLKHIGKGTGEFIKALMKEIPALLHLPVLIIMALAILSFCYGAGKSVHVLRHIGGPESEPPQALRPRDRRRQEEIDYRPDGGAGDADFHYRGQMGPTEQGPYAKTYEGRREILRERDVDLRFQTGNKSPEVLRAFDVPDAEAREHPTVVPSHKSPVLDTKPKETGGILGEGTPKESSTESSQSAKPVSGQDTSGNTEGSPAAEKAQLKSEAAGSPDQGSTYSPARGVAGPRGQDPVSSPCG",
# CP2S1_HUMAN
"MEATGTWALLLALALLLLLTLALSGTRARGHLPPGPTPLPLLGNLLQLRPGALYSGLMRLSKKYGPVFTIYLGPWRPVVVLVGQEAVREALGGQAEEFSGRGTVAMLEGTFDGHGVFFSNGERWRQLRKFTMLALRDLGMGKREGEELIQAEARCLVETFQGTEGRPFDPSLLLAQATSNVVCSLLFGLRFSYEDKEFQAVVRAAGGTLLGVSSQGGQTYEMFSWFLRPLPGPHKQLLHHVSTLAAFTVRQVQQHQGNLDASGPARDLVDAFLLKMAQEEQNPGTEFTNKNMLMTVIYLLFAGTMTVSTTVGYTLLLLMKYPHVQKWVREELNRELGAGQAPSLGDRTRLPYTDAVLHEAQRLLALVPMGIPRTLMRTTRFRGYTLPQGTEVFPLLGSILHDPNIFKHPEEFNPDRFLDADGRFRKHEAFLPFSLGKRVCLGEGLAKAELFLFFTTILQAFSLESPCPPDTLSLKPTVSGLFNIPPAFQLQVRPTDLHSTTQTR",
# SEC62_HUMAN
"MAERRRHKKRIQEVGEPSKEEKAVAKYLRFNCPTKSTNMMGHRVDYFIASKAVDCLLDSKWAKAKKGEEALFTTRESVVDYCNRLLKKQFFHRALKVMKMKYDKDIKKEKDKGKAESGKEEDKKSKKENIKDEKTKKEKEKKKDGEKEESKKEETPGTPKKKETKKKFKLEPHDDQVFLDGNEVYVWIYDPVHFKTFVMGLILVIAVIAATLFPLWPAEMRVGVYYLSVGAGCFVASILLLAVARCILFLIIWLITGGRHHFWFLPNLTADVGFIDSFRPLYTHEYKGPKADLKKDEKSETKKQQKSDSEEKSDSEKKEDEEGKVGPGNHGTEGSGGERHSDTDSDRREDDRSQHSSGNGNDFEMITKEELEQQTDGDCEEDEEEENDGETPKSSHEKS",
# GPER1_HUMAN
"MDVTSQARGVGLEMYPGTAQPAAPNTTSPELNLSHPLLGTALANGTGELSEHQQYVIGLFLSCLYTIFLFPIGFVGNILILVVNISFREKMTIPDLYFINLAVADLILVADSLIEVFNLHERYYDIAVLCTFMSLFLQVNMYSSVFFLTWMSFDRYIALARAMRCSLFRTKHHARLSCGLIWMASVSATLVPFTAVHLQHTDEACFCFADVREVQWLEVTLGFIVPFAIIGLCYSLIVRVLVRAHRHRGLRPRRQKALRMILAVVLVFFVCWLPENVFISVHLLQRTQPGAAPCKQSFRHAHPLTGHIVNLAAFSNSCLNPLIYSFLGETFRDKLRLYIEQKTNLPALNRFCHAALKAVIPDSTEQSDVRFSSAV",
# PMYT1_HUMAN
"MLERPPALAMPMPTEGTPPPLSGTPIPVPAYFRHAEPGFSLKRPRGLSRSLPPPPPAKGSIPISRLFPPRTPGWHQLQPRRVSFRGEASETLQSPGYDPSRPESFFQQSFQRLSRLGHGSYGEVFKVRSKEDGRLYAVKRSMSPFRGPKDRARKLAEVGSHEKVGQHPCCVRLEQAWEEGGILYLQTELCGPSLQQHCEAWGASLPEAQVWGYLRDTLLALAHLHSQGLVHLDVKPANIFLGPRGRCKLGDFGLLVELGTAGAGEVQEGDPRYMAPELLQGSYGTAADVFSLGLTILEVACNMELPHGGEGWQQLRQGYLPPEFTAGLSSELRSVLVMMLEPDPKLRATAEALLALPVLRQPRAWGVLWCMAAEALSRGWALWQALLALLCWLWHGLAHPASWLQPLGPPATPPGSPPCSLLLDSSLSSNWDDDSLGPSLSPEAVLARTVGSTSTPRSRCTPRDALDLSDINSEPPRGSFPSFEPRNLLSLFEDTLDPT",
# SGMR1_HUMAN
"MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLARQYAGLDHELAFSRLIVELRRLHPGHVLPDEELQWVFVNAGGWMGAMCLLHASLSEYVLLFGTALGSRGHSGRYWAEISDTIISGTFHQWREGTTKSEVFYPGETVVHGPGEATAVEWGPNTWMVEYGRGVIPSTLAFALADTVFSTQDFLTLFYTLRSYARGLRLELTTYLFGQDP",
# MGST2_HUMAN
"MAGNSILLAAVSILSACQQSYFALQVGKARLKYKVTPPAVTGSPEFERVFRAQQNCVEFYPIFIITLWMAGWYFNQVFATCLGLVYIYGRHLYFWGYSEAAKKRITGFRLSLGILALLTLLGALGIANSFLDEYLDLNIAKKLRRQF",
# RNF5_HUMAN
"MAAAEEEDGGPEGPNRERGGAGATFECNICLETAREAVVSVCGHLYCWPCLHQWLETRPERQECPVCKAGISREKVVPLYGRGSQKPQDPRLKTPPRPQGQRPAPESRGGFQPFGDTGGFHFSFGVGAFPFGFFTTVFNAHEPFRRGTGVDLGQGHPASSWQDSLFLFLAIFFFFWLLSI",
# PLCA_HUMAN
"MDLWPGAWMLLLLLFLLLLFLLPTLWFCSPSAKYFFKMAFYNGWILFLAVLAIPVCAVRGRNVENMKILRLMLLHIKYLYGIRVEVRGAHHFPPSQPYVVVSNHQSSLDLLGMMEVLPGRCVPIAKRELLWAGSAGLACWLAGVIFIDRKRTGDAISVMSEVAQTLLTQDVRVWVFPEGTRNHNGSMLPFKRGAFHLAVQAQVPIVPIVMSSYQDFYCKKERRFTSGQCQVRVLPPVPTEGLTPDDVPALADRVRHSMLTVFREISTDGRGGGDYLKKPGGGG",
# SELS_HUMAN
"MERQEESLSARPALETEGLRFLHTTVGSLLATYGWYIVFSCILLYVVFQKLSARLRALRQRQLDRAAAAVEPDVVVKRQEALAAARLKMQEELNAQVEKHKEKLKQLEEEKRRQKIEMWDSMQEGKSYKGNAKKPQEEDSPGPSTSSVLKRKSDRKPLRGGGYNPLSGEGGGACSWRPGRRGPSSGGUG",
# REEP2_HUMAN
"MVSWIISRLVVLIFGTLYPAYSSYKAVKTKNVKEYVKWMMYWIVFAFFTTAETLTDIVLSWFPFYFELKIAFVIWLLSPYTKGSSVLYRKFVHPTLSNKEKEIDEYITQARDKSYETMMRVGKRGLNLAANAAVTAAAKGVLSEKLRSFSMQDLTLIRDEDALPLQRPDGRLRPSPGSLLDTIEDLGDDPALSLRSSTNPADSRTEASEDDMGDKAPKRAKPIKKAPKAEPLASKTLKTRPKKKTSGGGDSA",
# SC22C_HUMAN
"MSVIFFACVVRVRDGLPLSASTDFYHTQDFLEWRRRLKSLALRLAQYPGRGSAEGCDFSIHFSSFGDVACMAICSCQCPAAMAFCFLETLWWEFTASYDTTCIGLASRPYAFLEFDSIIQKVKWHFNYVSSSQMECSLEKIQEELKLQPPAVLTLEDTDVANGVMNGHTPMHLEPAPNFRMEPVTALGILSLILNIMCAALNLIRGVHLAEHSLQVAHEEIGNILAFLVPFVACIFQCYLYLFYSPARTMKVVLMLLFICLGNMYLHGLRNLWQILFHIGVAFLSSYQILTRQLQEKQSDCGV",
# ADPGK_HUMAN
"MALWRGSAYAGFLALAVGCVFLLEPELPGSALRSLWSSLCLGPAPAPPGPVSPEGRLAAAWDALIVRPVRRWRRVAVGVNACVDVVLSGVKLLQALGLSPGNGKDHSILHSRNDLEEAFIHFMGKGAAAERFFSDKETFHDIAQVASEFPGAQHYVGGNAALIGQKFAANSDLKVLLCGPVGPKLHELLDDNVFVPPESLQEVDEFHLILEYQAGEEWGQLKAPHANRFIFSHDLSNGAMNMLEVFVSSLEEFQPDLVVLSGLHMMEGQSKELQRKRLLEVVTSISDIPTGIPVHLELASMTNRELMSSIVHQQVFPAVTSLGLNEQELLFLTQSASGPHSSLSSWNGVPDVGMVSDILFWILKEHGRSKSRASDLTRIHFHTLVYHILATVDGHWANQLAAVAAGARVAGTQACATETIDTSRVSLRAPQEFMTSHSEAGSRIVLNPNKPVVEWHREGISFHFTPVLVCKDPIRTVGLGDAISAEGLFYSEVHPHY",
# ESYT1_HUMAN
"MERSPGEGPSPSPMDQPSAPSDPTDQPPAAHAKPDPGSGGQPAGPGAAGEALAVLTSFGRRLLVLIPVYLAGAVGLSVGFVLFGLALYLGWRRVRDEKERSLRAARQLLDDEEQLTAKTLYMSHRELPAWVSFPDVEKAEWLNKIVAQVWPFLGQYMEKLLAETVAPAVRGSNPHLQTFTFTRVELGEKPLRIIGVKVHPGQRKEQILLDLNISYVGDVQIDVEVKKYFCKAGVKGMQLHGVLRVILEPLIGDLPFVGAVSMFFIRRPTLDINWTGMTNLLDIPGLSSLSDTMIMDSIAAFLVLPNRLLVPLVPDLQDVAQLRSPLPRGIIRIHLLAARGLSSKDKYVKGLIEGKSDPYALVRLGTQTFCSRVIDEELNPQWGETYEVMVHEVPGQEIEVEVFDKDPDKDDFLGRMKLDVGKVLQASVLDDWFPLQGGQGQVHLRLEWLSLLSDAEKLEQVLQWNWGVSSRPDPPSAAILVVYLDRAQDLPLKKGNKEPNPMVQLSIQDVTQESKAVYSTNCPVWEEAFRFFLQDPQSQELDVQVKDDSRALTLGALTLPLARLLTAPELILDQWFQLSSSGPNSRLYMKLVMRILYLDSSEICFPTVPGCPGAWDVDSENPQRGSSVDAPPRPCHTTPDSQFGTEHVLRIHVLEAQDLIAKDRFLGGLVKGKSDPYVKLKLAGRSFRSHVVREDLNPRWNEVFEVIVTSVPGQELEVEVFDKDLDKDDFLGRCKVRLTTVLNSGFLDEWLTLEDVPSGRLHLRLERLTPRPTAAELEEVLQVNSLIQTQKSAELAAALLSIYMERAEDLPLRKGTKHLSPYATLTVGDSSHKTKTISQTSAPVWDESASFLIRKPHTESLELQVRGEGTGVLGSLSLPLSELLVADQLCLDRWFTLSSGQGQVLLRAQLGILVSQHSGVEAHSHSYSHSSSSLSEEPELSGGPPHITSSAPELRQRLTHVDSPLEAPAGPLGQVKLTLWYYSEERKLVSIVHGCRSLRQNGRDPPDPYVSLLLLPDKNRGTKRRTSQKKRTLSPEFNERFEWELPLDEAQRRKLDVSVKSNSSFMSRERELLGKVQLDLAETDLSQGVARWYDLMDNKDKGSS",
# ALG1_HUMAN
"MAASCLVLLALCLLLPLLLLGGWKRWRRGRAARHVVAVVLGDVGRSPRMQYHALSLAMHGFSVTLLGFCNSKPHDELLQNNRIQIVGLTELQSLAVGPRVFQYGVKVVLQAMYLLWKLMWREPGAYIFLQNPPGLPSIAVCWFVGCLCGSKLVIDWHNYGYSIMGLVHGPNHPLVLLAKWYEKFFGRLSHLNLCVTNAMREDLADNWHIRAVTVYDKPASFFKETPLDLQHRLFMKLGSMHSPFRARSEPEDPVTERSAFTERDAGSGLVTRLRERPALLVSSTSWTEDEDFSILLAALEKFEQLTLDGHNLPSLVCVITGKGPLREYYSRLIHQKHFQHIQVCTPWLEAEDYPLLLGSADLGVCLHTSSSGLDLPMKVVDMFGCCLPVCAVNFKCLHELVKHEENGLVFEDSEELAAQLQMLFSNFPDPAGKLNQFRKNLRESQQLRWDESWVQTVLPLVMDT",
# G6PC3_HUMAN
"MESTLGAGIVIAEALQNQLAWLENVWLWITFLGDPKILFLFYFPAAYYASRRVGIAVLWISLITEWLNLIFKWFLFGDRPFWWVHESGYYSQAPAQVHQFPSSCETGPGSPSGHCMITGAALWPIMTALSSQVATRARSRWVRVMPSLAYCTFLLAVGLSRIFILAHFPHQVLAGLITGAVLGWLMTPRVPMERELSFYGLTALALMLGTSLIYWTLFTLGLDLSWSISLAFKWCERPEWIHVDSRPFASLSRDSGAALGLGIALHSPCYAQVRRAQLGNGQKIACLVLAMGLLGPLDWLGHPPQISLFYIFNFLKYTLWPCLVLALVPWAVHMFSAQEAPPIHSS",
# DERL1_HUMAN
"MSDIGDWFRSIPAITRYWFAATVAVPLVGKLGLISPAYLFLWPEAFLYRFQIWRPITATFYFPVGPGTGFLYLVNLYFLYQYSTRLETGAFDGRPADYLFMLLFNWICIVITGLAMDMQLLMIPLIMSVLYVWAQLNRDMIVSFWFGTRFKACYLPWVILGFNYIIGGSVINELIGNLVGHLYFFLMFRYPMDLGGRNFLSTPQFLYRWLPSRRGGVSGFGVPPASMRRAADQNGGGGRHNWGQGFRLGDQ",
# ALG12_HUMAN
"MAGKGSSGRRPLLLGLLVAVATVHLVICPYTKVEESFNLQATHDLLYHWQDLEQYDHLEFPGVVPRTFLGPVVIAVFSSPAVYVLSLLEMSKFYSQLIVRGVLGLGVIFGLWTLQKEVRRHFGAMVATMFCWVTAMQFHLMFYCTRTLPNVLALPVVLLALAAWLRHEWARFIWLSAFAIIVFRVELCLFLGLLLLLALGNRKVSVVRALRHAVPAGILCLGLTVAVDSYFWRQLTWPEGKVLWYNTVLNKSSNWGTSPLLWYFYSALPRGLGCSLLFIPLGLVDRRTHAPTVLALGFMALYSLLPHKELRFIIYAFPMLNITAARGCSYLLNNYKKSWLYKAGSLLVIGHLVVNAAYSATALYVSHFNYPGGVAMQRLHQLVPPQTDVLLHIDVAAAQTGVSRFLQVNSAWRYDKREDVQPGTGMLAYTHILMEAAPGLLALYRDTHRVLASVVGTTGVSLNLTQLPPFNVHLQTKLVLLERLPRPS",
# TMED9_HUMAN
"MAVELGVLLVRPRPGTGLGRVMRTLLLVLWLATRGSALYFHIGETEKKCFIEEIPDETMVIGNYRTQLYDKQREEYQPATPGLGMFVEVKDPEDKVILARQYGSEGRFTFTSHTPGEHQICLHSNSTKFSLFAGGMLRVHLDIQVGEHANDYAEIAAKDKLSELQLRVRQLVEQVEQIQKEQNYQRWREERFRQTSESTNQRVLWWSILQTLILVAIGVWQMRHLKSFFEAKKLV",
# ELOV1_HUMAN
"MEAVVNLYQEVMKHADPRIQGYPLMGSPLLMTSILLTYVYFVLSLGPRIMANRKPFQLRGFMIVYNFSLVALSLYIVYEFLMSGWLSTYTWRCDPVDYSNSPEALRMVRVAWLFLFSKFIELMDTVIFILRKKDGQVTFLHVFHHSVLPWSWWWGVKIAPGGMGSFHAMINSSVHVIMYLYYGLSAFGPVAQPYLWWKKHMTAIQLIQFVLVSLHISQYYFMSSCNYQYPVIIHLIWMYGTIFFMLFSNFWYHSYTKGKRLPRALQQNGAPGIAKVKAN",
# EBPL_HUMAN
"MGAEWELGAEAGGSLLLCAALLAAGCALGLRLGRGQGAADRGALIWLCYDALVHFALEGPFVYLSLVGNVANSDGLIASLWKEYGKADARWVYFDPTIVSVEILTVALDGSLALFLIYAIVKEKYYRHFLQITLCVCELYGCWMTFLPEWLTRSPNLNTSNWLYCWLYLFFFNGVWVLIPGLLLWQSWLELKKMHQKETSSVKKFQ",
# UDB28_HUMAN
"MALKWTSVLLLIHLGCYFSSGSCGKVLVWTGEYSHWMNMKTILKELVQRGHEVTVLASSASILFDPNDAFTLKLEVYPTSLTKTEFENIIMQQVKRWSDIQKDSFWLYFSQEQEILWEFHDIFRNFCKDVVSNKKVMKKLQESRFDIIFADAFFPCGELLAALLNIPFVYSLCFTPGYTIERHSGGLIFPPSYIPVVMSKLSDQMTFMERVKNMIYVLYFDFWFQMCDMKKWDQFYSEVLGRPTTLFETMGKADIWLMRNSWSFQFPHPFLPNIDFVGGLHCKPAKPLPKEMEEFVQSSGENGVVVFSLGSVISNMTAERANVIATALAKIPQKVLWRFDGNKPDALGLNTRLYKWIPQNDLLGLPKTRAFITHGGANGIYEAIYHGIPMVGIPLFWDQPDNIAHMKAKGAAVRLDFHTMSSTDLLNALKTVINDPSYKENVMKLSIIQHDQPVKPLHRAVFWIEFVMCHKGAKHLRVAARDLTWFQYHSLDVIGFLLACVATVIFVVTKFCLFCFWKFARKGKKGKRD",
# LNP_HUMAN
"MGGLFSRWRTKPSTVEVLESIDKEIQALEEFREKNQRLQKLWVGRLILYSSVLYLFTCLIVYLWYLPDEFTARLAMTLPFFAFPLIIWSIRTVIIFFFSKRTERNNEALDDLKSQRKKILEEVMEKETYKTAKLILERFDPDSKKAKECEPPSAGAAVTARPGQEIRQRTAAQRNLSPTPASPNQGPPPQVPVSPGPPKDSSAPGGPPERTVTPALSSNVLPRHLGSPATSVPGMGLHPPGPPLARPILPRERGALDRIVEYLVGDGPQNRYALICQQCFSHNGMALKEEFEYIAFRCAYCFFLNPARKTRPQAPRLPEFSFEKRQVVEGSSSVGPLPSGSVLSSDNQFNEESLEHDVLDDNTEQTDDKIPATEQTNQVIEKASDSEEPEEKQETENEEASVIETNSTVPGADSIPDPELSGESLTAE",
# DERL2_HUMAN
"MAYQSLRLEYLQIPPVSRAYTTACVLTTAAVQLELITPFQLYFNPELIFKHFQIWRLITNFLFFGPVGFNFLFNMIFLYRYCRMLEEGSFRGRTADFVFMFLFGGFLMTLFGLFVSLVFLGQAFTIMLVYVWSRRNPYVRMNFFGLLNFQAPFLPWVLMGFSLLLGNSIIVDLLGIAVGHIYFFLEDVFPNQPGGIRILKTPSILKAIFDTPDEDPNYNPLPEERPGGFAWGEGQRLGG",
# ELOV4_HUMAN
"MGLLDSEPGSVLNVVSTALNDTVEFYRWTWSIADKRVENWPLMQSPWPTLSISTLYLLFVWLGPKWMKDREPFQMRLVLIIYNFGMVLLNLFIFRELFMGSYNAGYSYICQSVDYSNNVHEVRIAAALWWYFVSKGVEYLDTVFFILRKKNNQVSFLHVYHHCTMFTLWWIGIKWVAGGQAFFGAQLNSFIHVIMYSYYGLTAFGPWIQKYLWWKRYLTMLQLIQFHVTIGHTALSLYTDCPFPKWMHWALIAYAISFIFLFLNFYIRTYKEPKKPKAGKTAMNGISANGVSKSEKQLMIENGKKQKNGKAKGD",
# MAGT1_HUMAN
"MAARWRFWCVSVTMVVALLIVCDVPSASAQRKKEMVLSEKVSQLMEWTNKRPVIRMNGDKFRRLVKAPPRNYSVIVMFTALQLHRQCVVCKQADEEFQILANSWRYSSAFTNRIFFAMVDFDEGSDVFQMLNMNSAPTFINFPAKGKPKRGDTYELQVRGFSAEQIARWIADRTDVNIRVIRPPNYAGPLMLGLLLAVIGGLVYLRRSNMEFLFNKTGWAFAALCFVLAMTSGQMWNHIRGPPYAHKNPHTGHVNYIHGSSQAQFVAETHIVLLFNGGVTLGMVLLCEAATSDMDIGKRKIMCVAGIGLVVLFFSWMLSIFRSKYHGYPYSFLMS",
# UN93B_HUMAN
"MEAEPPLYPMAGAAGPQGDEDLLGVPDGPEAPLDELVGAYPNYNEEEEERRYYRRKRLGVLKNVLAASAGGMLTYGVYLGLLQMQLILHYDETYREVKYGNMGLPDIDSKMLMGINVTPIAALLYTPVLIRFFGTKWMMFLAVGIYALFVSTNYWERYYTLVPSAVALGMAIVPLWASMGNYITRMAQKYHEYSHYKEQDGQGMKQRPPRGSHAPYLLVFQAIFYSFFHLSFACAQLPMIYFLNHYLYDLNHTLYNVQSCGTNSHGILSGFNKTVLRTLPRSGNLIVVESVLMAVAFLAMLLVLGLCGAAYRPTEEIDLRSVGWGNIFQLPFKHVRDYRLRHLVPFFIYSGFEVLFACTGIALGYGVCSVGLERLAYLLVAYSLGASAASLLGLLGLWLPRPVPLVAGAGVHLLLTFILFFWAPVPRVLQHSWILYVAAALWGVGSALNKTGLSTLLGILYEDKERQDFIFTIYHWWQAVAIFTVYLGSSLHMKAKLAVLLVTLVAAAVSYLRMEQKLRRGVAPRQPRIPRPQHKVRGYRYLEEDNSDESDAEGEHGDGAEEEAPPAGPRPGPEPAGLGRRPCPYEQAQGGDGPEEQ",
# PORCN_HUMAN
"MATFSRQEFFQQLLQGCLLPTAQQGLDQIWLLLAICLACRLLWRLGLPSYLKHASTVAGGFFSLYHFFQLHMVWVVLLSLLCYLVLFLCRHSSHRGVFLSVTILIYLLMGEMHMVDTVTWHKMRGAQMIVAMKAVSLGFDLDRGEVGTVPSPVEFMGYLYFVGTIVFGPWISFHSYLQAVQGRPLSCRWLQKVARSLALALLCLVLSTCVGPYLFPYFIPLNGDRLLRNKKRKARGTMVRWLRAYESAVSFHFSNYFVGFLSEATATLAGAGFTEEKDHLEWDLTVSKPLNVELPRSMVEVVTSWNLPMSYWLNNYVFKNALRLGTFSAVLVTYAASALLHGFSFHLAAVLLSLAFITYVEHVLRKRLARILSACVLSKRCPPDCSHQHRLGLGVRALNLLFGALAIFHLAYLGSLFDVDVDDTTEEQGYGMAYTVHKWSELSWASHWVTFGCWIFYRLIG",
# ELOV6_HUMAN
"MNMSVLTLQEYEFEKQFNENEAIQWMQENWKKSFLFSALYAAFIFGGRHLMNKRAKFELRKPLVLWSLTLAVFSIFGALRTGAYMVYILMTKGLKQSVCDQGFYNGPVSKFWAYAFVLSKAPELGDTIFIILRKQKLIFLHWYHHITVLLYSWYSYKDMVAGGGWFMTMNYGVHAVMYSYYALRAAGFRVSRKFAMFITLSQITQMLMGCVVNYLVFCWMQHDQCHSHFQNIFWSSLMYLSYLVLFCHFFFEAYIGKMRKTTKAE",
# REEP4_HUMAN
"MVSWMICRLVVLVFGMLCPAYASYKAVKTKNIREYVRWMMYWIVFALFMAAEIVTDIFISWFPFYYEIKMAFVLWLLSPYTKGASLLYRKFVHPSLSRHEKEIDAYIVQAKERSYETVLSFGKRGLNIAASAAVQAATKSQGALAGRLRSFSMQDLRSISDAPAPAYHDPLYLEDQVSHRRPPIGYRAGGLQDSDTEDECWSDTEAVPRAPARPREKPLIRSQSLRVVKRKPPVREGTSRSLKVRTRKKTVPSDVDS",
# RETR1_HUMAN
"MASPAPPEHAEEGCPAPAAEEQAPPSPPPPQASPAERQQQEEEAQEAGAAEGAGLQVEEAAGRAAAAVTWLLGEPVLWLGCRADELLSWKRPLRSLLGFVAANLLFWFLALTPWRVYHLISVMILGRVIMQIIKDMVLSRTRGAQLWRSLSESWEVINSKPDERPRLSHCIAESWMNFSIFLQEMSLFKQQSPGKFCLLVCSVCTFFTILGSYIPGVILSYLLLLCAFLCPLFKCNDIGQKIYSKIKSVLLKLDFGIGEYINQKKRERSEADKEKSHKDDSELDFSALCPKISLTVAAKELSVSDTDVSEVSWTDNGTFNLSEGYTPQTDTSDDLDRPSEEVFSRDLSDFPSLENGMGTNDEDELSLGLPTELKRKKEQLDSGHRPSKETQSAAGLTLPLNSDQTFHLMSNLAGDVITAAVTAAIKDQLEGVQQALSQAAPIPEEDTDTEEGDDFELLDQSELDQIESELGLTQDQEAEAQQNKKSSGFLSNLLGGH",
# ZDHC6_HUMAN
"MGTFCSVIKFENLQELKRLCHWGPIIALGVIAICSTMAMIDSVLWYWPLHTTGGSVNFIMLINWTVMILYNYFNAMFVGPGFVPLGWKPEISQDTMYLQYCKVCQAYKAPRSHHCRKCNRCVMKMDHHCPWINNCCGYQNHASFTLFLLLAPLGCIHAAFIFVMTMYTQLYHRLSFGWNTVKIDMSAARRDPLPIVPFGLAAFATTLFALGLALGTTIAVGMLFFIQMKIILRNKTSIESWIEEKAKDRIQYYQLDEVFVFPYDMGSRWRNFKQVFTWSGVPEGDGLEWPVREGCHQYSLTIEQLKQKADKRVRSVRYKVIEDYSGACCPLNKGIKTFFTSPCTEEPRIQLQKGEFILATRGLRYWLYGDKILDDSFIEGVSRIRGWFPRKCVEKCPCDAETDQAPEGEKKNR",
# ALG9_HUMAN
"MASRGARQRLKGSGASSGDTAPAADKLRELLGSREAGGAEHRTELSGNKAGQVWAPEGSTAFKCLLSARLCAALLSNISDCDETFNYWEPTHYLIYGEGFQTWEYSPAYAIRSYAYLLLHAWPAAFHARILQTNKILVFYFLRCLLAFVSCICELYFYKAVCKKFGLHVSRMMLAFLVLSTGMFCSSSAFLPSSFCMYTTLIAMTGWYMDKTSIAVLGVAAGAILGWPFSAALGLPIAFDLLVMKHRWKSFFHWSLMALILFLVPVVVIDSYYYGKLVIAPLNIVLYNVFTPHGPDLYGTEPWYFYLINGFLNFNVAFALALLVLPLTSLMEYLLQRFHVQNLGHPYWLTLAPMYIWFIIFFIQPHKEERFLFPVYPLICLCGAVALSALQKCYHFVFQRYRLEHYTVTSNWLALGTVFLFGLLSFSRSVALFRGYHGPLDLYPEFYRIATDPTIHTVPEGRPVNVCVGKEWYRFPSSFLLPDNWQLQFIPSEFRGQLPKPFAEGPLATRIVPTDMNDQNLEEPSRYIDISKCHYLVDLDTMRETPREPKYSSNKEEWISLAYRPFLDASRSSKLLRAFYVPFLSDQYTVYVNYTILKPRKAKQIRKKSGG",
# SR5A3_HUMAN
"MAPWAEAEHSALNPLRAVWLTLTAAFLLTLLLQLLPPGLLPGCAIFQDLIRYGKTKCGEPSRPAACRAFDVPKRYFSHFYIISVLWNGFLLWCLTQSLFLGAPFPSWLHGLLRILGAAQFQGGELALSAFLVLVFLWLHSLRRLFECLYVSVFSNVMIHVVQYCFGLVYYVLVGLTVLSQVPMDGRNAYITGKNLLMQARWFHILGMMMFIWSSAHQYKCHVILGNLRKNKAGVVIHCNHRIPFGDWFEYVSSPNYLAELMIYVSMAVTFGFHNLTWWLVVTNVFFNQALSAFLSHQFYKSKFVSYPKHRKAFLPFLF",
# ZDH11_HUMAN
"MDTRSGSQCSVTPEAILNNEKLVLPPRISRVNGWSLPLHYFQVVTWAVFVGLSSATFGIFIPFLPHAWKYIAYVVTGGIFSFHLVVHLIASCIDPADSNVRLMKNYSQPMPLFDRSKHAHVIQNQFCHLCKVTVNKKTKHCISCNKCVSGFDHHCKWINNCVGSRNYWFFFSTVASATAGMLCLIAILLYVLVQYLVNPGVLRTDPRYEDVKNMNTWLLFLPLFPVQVQTLIVVIIGMLVLLLDFLGLVHLGQLLIFHIYLKAKKMTTFEYLINNRKEESSKHQAVRKDPYVQMDKGVLQQGAGALGSSAQGVKAKSSLLIHKHLCHFCTSVNQDGDSTAREGDEDPCPSALGAKARNSRLICRRLCQFSTRVHPDGGSMAQEADDAPSISTLGLQQETTEPMKTDSAESED",
# REEP1_HUMAN
"MVSWIISRLVVLIFGTLYPAYYSYKAVKSKDIKEYVKWMMYWIIFALFTTAETFTDIFLCWFPFYYELKIAFVAWLLSPYTKGSSLLYRKFVHPTLSSKEKEIDDCLVQAKDRSYDALVHFGKRGLNVAATAAVMAASKGQGALSERLRSFSMQDLTTIRGDGAPAPSGPPPPGSGRASGKHGQPKMSRSASESASSSGTA",
# CERS4_HUMAN
"MLSSFNEWFWQDRFWLPPNVTWTELEDRDGRVYPHPQDLLAALPLALVLLAMRLAFERFIGLPLSRWLGVRDQTRRQVKPNATLEKHFLTEGHRPKEPQLSLLAAQCGLTLQQTQRWFRRRRNQDRPQLTKKFCEASWRFLFYLSSFVGGLSVLYHESWLWAPVMCWDRYPNQTLKPSLYWWYLLELGFYLSLLIRLPFDVKRKDFKEQVIHHFVAVILMTFSYSANLLRIGSLVLLLHDSSDYLLEACKMVNYMQYQQVCDALFLIFSFVFFYTRLVLFPTQILYTTYYESISNRGPFFGYYFFNGLLMLLQLLHVFWSCLILRMLYSFMKKGQMEKDIRSDVEESDSSEEAAAAQEPLQLKNGAAGGPRPAPTDGPRSRVAGRLTNRHTTAT",
# UD17_HUMAN
"MARAGWTGLLPLYVCLLLTCGFAKAGKLLVVPMDGSHWFTMQSVVEKLILRGHEVVVVMPEVSWQLGRSLNCTVKTYSTSYTLEDQDREFMVFADARWTAPLRSAFSLLTSSSNGIFDLFFSNCRSLFNDRKLVEYLKESCFDAVFLDPFDACGLIVAKYFSLPSVVFARGIFCHYLEEGAQCPAPLSYVPRLLLGFSDAMTFKERVWNHIMHLEEHLFCPYFFKNVLEIASEILQTPVTAYDLYSHTSIWLLRTDFVLEYPKPVMPNMIFIGGINCHQGKPVPMEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# UD110_HUMAN
"MARAGWTSPVPLCVCLLLTCGFAEAGKLLVVPMDGSHWFTMQSVVEKLILRGHEVVVVMPEVSWQLERSLNCTVKTYSTSYTLEDQNREFMVFAHAQWKAQAQSIFSLLMSSSSGFLDLFFSHCRSLFNDRKLVEYLKESSFDAVFLDPFDTCGLIVAKYFSLPSVVFTRGIFCHHLEEGAQCPAPLSYVPNDLLGFSDAMTFKERVWNHIVHLEDHLFCQYLFRNALEIASEILQTPVTAYDLYSHTSIWLLRTDFVLDYPKPVMPNMIFIGGINCHQGKPLPMEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# UD18_HUMAN
"MARTGWTSPIPLCVSLLLTCGFAEAGKLLVVPMDGSHWFTMQSVVEKLILRGHEVVVVMPEVSWQLGKSLNCTVKTYSTSYTLEDLDREFMDFADAQWKAQVRSLFSLFLSSSNGFFNLFFSHCRSLFNDRKLVEYLKESSFDAVFLDPFDACGLIVAKYFSLPSVVFARGIACHYLEEGAQCPAPLSYVPRILLGFSDAMTFKERVRNHIMHLEEHLFCQYFSKNALEIASEILQTPVTAYDLYSHTSIWLLRTDFVLDYPKPVMPNMIFIGGINCHQGKPLPMEFEAYINASGEHGIVVFSLGSMVSEIPEKKAMAIADALGKIPQTVLWRYTGTRPSNLANNTILVKWLPQNDLLGHPMTRAFITHAGSHGVYESICNGVPMVMMPLFGDQMDNAKRMETKGAGVTLNVLEMTSEDLENALKAVINDKSYKENIMRLSSLHKDRPVEPLDLAVFWVEFVMRHKGAPHLRPAAHDLTWYQYHSLDVIGFLLAVVLTVAFITFKCCAYGYRKCLGKKGRVKKAHKSKTH",
# ELOV3_HUMAN
"MVTAMNVSHEVNQLFQPYNFELSKDMRPFFEEYWATSFPIALIYLVLIAVGQNYMKERKGFNLQGPLILWSFCLAIFSILGAVRMWGIMGTVLLTGGLKQTVCFINFIDNSTVKFWSWVFLLSKVIELGDTAFIILRKRPLIFIHWYHHSTVLVYTSFGYKNKVPAGGWFVTMNFGVHAIMYTYYTLKAANVKPPKMLPMLITSLQILQMFVGAIVSILTYIWRQDQGCHTTMEHLFWSFILYMTYFILFAHFFCQTYIRPKVKAKTKSQ",
# ZFYV1_HUMAN
"MSAQTSPAEKGLNPGLMCQESYACSGTDEAIFECDECCSLQCLRCEEELHRQERLRNHERIRLKPGHVPYCDLCKGLSGHLPGVRQRAIVRCQTCKINLCLECQKRTHSGGNKRRHPVTVYNVSNLQESLEAEEMDEETKRKKMTEKVVSFLLVDENEEIQVTNEEDFIRKLDCKPDQHLKVVSIFGNTGDGKSHTLNHTFFYGREVFKTSPTQESCTVGVWAAYDPVHKVAVIDTEGLLGATVNLSQRTRLLLKVLAISDLVIYRTHADRLHNDLFKFLGDASEAYLKHFTKELKATTARCGLDVPLSTLGPAVIIFHETVHTQLLGSDHPSEVPEKLIQDRFRKLGRFPEAFSSIHYKGTRTYNPPTDFSGLRRALEQLLENNTTRSPRHPGVIFKALKALSDRFSGEIPDDQMAHSSFFPDEYFTCSSLCLSCGVGCKKSMNHGKEGVPHEAKSRCRYSHQYDNRVYTCKACYERGEEVSVVPKTSASTDSPWMGLAKYAWSGYVIECPNCGVVYRSRQYWFGNQDPVDTVVRTEIVHVWPGTDGFLKDNNNAAQRLLDGMNFMAQSVSELSLGPTKAVTSWLTDQIAPAYWRPNSQILSCNKCATSFKDNDTKHHCRACGEGFCDSCSSKTRPVPERGWGPAPVRVCDNCYEARNVQLAVTEAQVDDEGGTLIARKVGEAVQNTLGAVVTAIDIPLGLVKDAARPAYWVPDHEILHCHNCRKEFSIKLSKHHCRACGQGFCDECSHDRRAVPSRGWDHPVRVCFNCNKKPGDL",
# RDH14_HUMAN
"MAVATAAAVLAALGGALWLAARRFVGPRVQRLRRGGDPGLMHGKTVLITGANSGLGRATAAELLRLGARVIMGCRDRARAEEAAGQLRRELRQAAECGPEPGVSGVGELIVRELDLASLRSVRAFCQEMLQEEPRLDVLINNAGIFQCPYMKTEDGFEMQFGVNHLGHFLLTNLLLGLLKSSAPSRIVVVSSKLYKYGDINFDDLNSEQSYNKSFCYSRSKLANILFTRELARRLEGTNVTVNVLHPGIVRTNLGRHIHIPLLVKPLFNLVSWAFFKTPVEGAQTSIYLASSPEVEGVSGRYFGDCKEEELLPKAMDESVARKLWDISEVMVGLLK",
# HHATL_HUMAN
"MGIKTALPAAELGLYSLVLSGALAYAGRGLLEASQDGAHRKAFRESVRPGWEYIGRKMDVADFEWVMWFTSFRNVIIFALSGHVLFAKLCTMVAPKLRSWMYAVYGALAVMGTMGPWYLLLLLGHCVGLYVASLLGQPWLCLGLGLASLASFKMDPLISWQSGFVTGTFDLQEVLFHGGSSFTVLRCTSFALESCAHPDRHYSLADLLKYNFYLPFFFFGPIMTFDRFHAQVSQVEPVRREGELWHIRAQAGLSVVAIMAVDIFFHFFYILTIPSDLKFANRLPDSALAGLAYSNLVYDWVKAAVLFGVVNTVACLDHLDPPQPPKCITALYVFAETHFDRGINDWLCKYVYNHIGGEHSAVIPELAATVATFAITTLWLGPCDIVYLWSFLNCFGLNFELWMQKLAEWGPLARIEASLSVQMSRRVRALFGAMNFWAIIMYNLVSLNSLKFTELVARRLLLTGFPQTTLSILFVTYCGVQLVKERERTLALEEEQKQDKEKPE",
# ABCB6_HUMAN
"MVTVGNYCEAEGPVGPAWMQDGLSPCFFFTLVPSTRMALGTLALVLALPCRRRERPAGADSLSWGAGPRISPYVLQLLLATLQAALPLAGLAGRVGTARGAPLPSYLLLASVLESLAGACGLWLLVVERSQARQRLAMGIWIKFRHSPGLLLLWTVAFAAENLALVSWNSPQWWWARADLGQQVQFSLWVLRYVVSGGLFVLGLWAPGLRPQSYTLQVHEEDQDVERSQVRSAAQQSTWRDFGRKLRLLSGYLWPRGSPALQLVVLICLGLMGLERALNVLVPIFYRNIVNLLTEKAPWNSLAWTVTSYVFLKFLQGGGTGSTGFVSNLRTFLWIRVQQFTSRRVELLIFSHLHELSLRWHLGRRTGEVLRIADRGTSSVTGLLSYLVFNVIPTLADIIIGIIYFSMFFNAWFGLIVFLCMSLYLTLTIVVTEWRTKFRRAMNTQENATRARAVDSLLNFETVKYYNAESYEVERYREAIIKYQGLEWKSSASLVLLNQTQNLVIGLGLLAGSLLCAYFVTEQKLQVGDYVLFGTYIIQLYMPLNWFGTYYRMIQTNFIDMENMFDLLKEETEVKDLPGAGPLRFQKGRIEFENVHFSYADGRETLQDVSFTVMPGQTLALVGPSGAGKSTILRLLFRFYDISSGCIRIDGQDISQVTQASLRSHIGVVPQDTVLFNDTIADNIRYGRVTAGNDEVEAAAQAAGIHDAIMAFPEGYRTQVGERGLKLSGGEKQRVAIARTILKAPGIILLDEATSALDTSNERAIQASLAKVCANRTTIVVAHRLSTVVNADQILVIKDGCIVERGRHEALLSRGGVYADMWQLQQGQEETSEDTKPQTMER",
# ISK5_HUMAN
"MKIATVSVLLPLALCLIQDAASKNEDQEMCHEFQAFMKNGKLFCPQDKKFFQSLDGIMFINKCATCKMILEKEAKSQKRARHLARAPKATAPTELNCDDFKKGERDGDFICPDYYEAVCGTDGKTYDNRCALCAENAKTGSQIGVKSEGECKSSNPEQDVCSAFRPFVRDGRLGCTRENDPVLGPDGKTHGNKCAMCAELFLKEAENAKREGETRIRRNAEKDFCKEYEKQVRNGRLFCTRESDPVRGPDGRMHGNKCALCAEIFKQRFSEENSKTDQNLGKAEEKTKVKREIVKLCSQYQNQAKNGILFCTRENDPIRGPDGKMHGNLCSMCQAYFQAENEEKKKAEARARNKRESGKATSYAELCSEYRKLVRNGKLACTRENDPIQGPDGKVHGNTCSMCEVFFQAEEEEKKKKEGKSRNKRQSKSTASFEELCSEYRKSRKNGRLFCTRENDPIQGPDGKMHGNTCSMCEAFFQQEERARAKAKREAAKEICSEFRDQVRNGTLICTREHNPVRGPDGKMHGNKCAMCASVFKLEEEEKKNDKEEKGKVEAEKVKREAVQELCSEYRHYVRNGRLPCTRENDPIEGLDGKIHGNTCSMCEAFFQQEAKEKERAEPRAKVKREAEKETCDEFRRLLQNGKLFCTRENDPVRGPDGKTHGNKCAMCKAVFQKENEERKRKEEEDQRNAAGHGSSGGGGGNTQDECAEYREQMKNGRLSCTRESDPVRDADGKSYNNQCTMCKAKLEREAERKNEYSRSRSNGTGSESGKDTCDEFRSQMKNGKLICTRESDPVRGPDGKTHGNKCTMCKEKLEREAAEKKKKEDEDRSNTGERSNTGERSNDKEDLCREFRSMQRNGKLICTRENNPVRGPYGKMHINKCAMCQSIFDREANERKKKDEEKSSSKPSNNAKDECSEFRNYIRNNELICPRENDPVHGADGKFYTNKCYMCRAVFLTEALERAKLQEKPSHVRASQEEDSPDSFSSLDSEMCKDYRVLPRIGYLCPKDLKPVCGDDGQTYNNPCMLCHENLIRQTNTHIRSTGKCEESSTPGTTAASMPPSDE",
# RTN4_HUMAN
"MEDLDQSPLVSSSDSPPRPQPAFKYQFVREPEDEEEEEEEEEEDEDEDLEELEVLERKPAAGLSAAPVPTAPAAGAPLMDFGNDFVPPAPRGPLPAAPPVAPERQPSWDPSPVSSTVPAPSPLSAAAVSPSKLPEDDEPPARPPPPPPASVSPQAEPVWTPPAPAPAAPPSTPAAPKRRGSSGSVDETLFALPAASEPVIRSSAENMDLKEQPGNTISAGQEDFPSVLLETAASLPSLSPLSAASFKEHEYLGNLSTVLPTEGTLQENVSEASKEVSEKAKTLLIDRDLTEFSELEYSEMGSSFSVSPKAESAVIVANPREEIIVKNKDEEEKLVSNNILHNQQELPTALTKLVKEDEVVSSEKAKDSFNEKRVAVEAPMREEYADFKPFERVWEVKDSKEDSDMLAAGGKIESNLESKVDKKCFADSLEQTNHEKDSESSNDDTSFPSTPEGIKDRSGAYITCAPFNPAATESIATNIFPLLGDPTSENKTDEKKIEEKKAQIVTEKNTSTKTSNPFLVAAQDSETDYVTTDNLTKVTEEVVANMPEGLTPDLVQEACESELNEVTGTKIAYETKMDLVQTSEVMQESLYPAAQLCPSFEESEATPSPVLPDIVMEAPLNSAVPSAGASVIQPSSSPLEASSVNYESIKHEPENPPPYEEAMSVSLKKVSGIKEEIKEPENINAALQETEAPYISIACDLIKETKLSAEPAPDFSDYSEMAKVEQPVPDHSELVEDSSPDSEPVDLFSDDSIPDVPQKQDETVMLVKESLTETSFESMIEYENKEKLSALPPEGGKPYLESFKLSLDNTKDTLLPDEVSTLSKKEKIPLQMEELSTAVYSNDDLFISKEAQIRETETFSDSSPIEIIDEFPTLISSKTDSFSKLAREYTDLEVSHKSEIANAPDGAGSLPCTELPHDLSLKNIQPKVEEKISFSDDFSKNGSATSKVLLLPPDVSALATQAEIESIVKPKVLVKEAEKKLPSDTEKEDRSPSAIFSAELSKTSVVDLLYWRDIKKTGVVFGASLFLLLSLTVFSIVSVTAYIALALLSVTISFRIYKGVIQAIQKSDEGHPFRAYLESEVAISEELVQKYSNSALGHVNCTIKELRRLFLVDDLVDSLKFAVLMWVFTYVGALFNGLTLLILALISLFSVPVIYERHQAQIDHYLGLANKNVKDAMAKIQAKIPGLKRKAE",
# TLR9_HUMAN
"MGFCRSALHPLSLLVQAIMLAMTLALGTLPAFLPCELQPHGLVNCNWLFLKSVPHFSMAAPRGNVTSLSLSSNRIHHLHDSDFAHLPSLRHLNLKWNCPPVGLSPMHFPCHMTIEPSTFLAVPTLEELNLSYNNIMTVPALPKSLISLSLSHTNILMLDSASLAGLHALRFLFMDGNCYYKNPCRQALEVAPGALLGLGNLTHLSLKYNNLTVVPRNLPSSLEYLLLSYNRIVKLAPEDLANLTALRVLDVGGNCRRCDHAPNPCMECPRHFPQLHPDTFSHLSRLEGLVLKDSSLSWLNASWFRGLGNLRVLDLSENFLYKCITKTKAFQGLTQLRKLNLSFNYQKRVSFAHLSLAPSFGSLVALKELDMHGIFFRSLDETTLRPLARLPMLQTLRLQMNFINQAQLGIFRAFPGLRYVDLSDNRISGASELTATMGEADGGEKVWLQPGDLAPAPVDTPSSEDFRPNCSTLNFTLDLSRNNLVTVQPEMFAQLSHLQCLRLSHNCISQAVNGSQFLPLTGLQVLDLSHNKLDLYHEHSFTELPRLEALDLSYNSQPFGMQGVGHNFSFVAHLRTLRHLSLAHNNIHSQVSQQLCSTSLRALDFSGNALGHMWAEGDLYLHFFQGLSGLIWLDLSQNRLHTLLPQTLRNLPKSLQVLRLRDNYLAFFKWWSLHFLPKLEVLDLAGNQLKALTNGSLPAGTRLRRLDVSCNSISFVAPGFFSKAKELRELNLSANALKTVDHSWFGPLASALQILDVSANPLHCACGAAFMDFLLEVQAAVPGLPSRVKCGSPGQLQGLSIFAQDLRLCLDEALSWDCFALSLLAVALGLGVPMLHHLCGWDLWYCFHLCLAWLPWRGRQSGRDEDALPYDAFVVFDKTQSAVADWVYNELRGQLEECRGRWALRLCLEERDWLPGKTLFENLWASVYGSRKTLFVLAHTDRVSGLLRASFLLAQQRLLEDRKDVVVLVILSPDGRRSRYVRLRQRLCRQSVLLWPHQPSGQRSFWAQLGMALTRDNHHFYNRNFCQGPTAE",
# PLCC_HUMAN
"MGLLAFLKTQFVLHLLVGFVFVVSGLVINFVQLCTLALWPVSKQLYRRLNCRLAYSLWSQLVMLLEWWSCTECTLFTDQATVERFGKEHAVIILNHNFEIDFLCGWTMCERFGVLGSSKVLAKKELLYVPLIGWTWYFLEIVFCKRKWEEDRDTVVEGLRRLSDYPEYMWFLLYCEGTRFTETKHRVSMEVAAAKGLPVLKYHLLPRTKGFTTAVKCLRGTVAAVYDVTLNFRGNKNPSLLGILYGKKYEADMCVRRFPLEDIPLDEKEAAQWLHKLYQEKDALQEIYNQKGMFPGEQFKPARRPWTLLNFLSWATILLSPLFSFVLGVFASGSPLLILTFLGFVGAASFGVRRLIGVTEIEKGSSYGNQEFKKKE",
# CLN6_HUMAN
"MEATRRRQHLGATGGPGAQLGASFLQARHGSVSADEAARTAPFHLDLWFYFTLQNWVLDFGRPIAMLVFPLEWFPLNKPSVGDYFHMAYNVITPFLLLKLIERSPRTLPRSITYVSIIIFIMGASIHLVGDSVNHRLLFSGYQHHLSVRENPIIKNLKPETLIDSFELLYYYDEYLGHCMWYIPFFLILFMYFSGCFTASKAESLIPGPALLLVAPSGLYYWYLVTEGQIFILFIFTFFAMLALVLHQKRKRLFLDSNGLFLFSSFALTLLLVALWVAWLWNDPVLRKKYPGVIYVPEPWAFYTLHVSSRH",
# MARH5_HUMAN
"MPDQALQQMLDRSCWVCFATDEDDRTAEWVRPCRCRGSTKWVHQACLQRWVDEKQRGNSTARVACPQCNAEYLIVFPKLGPVVYVLDLADRLISKACPFAAAGIMVGSIYWTAVTYGAVTVMQVVGHKEGLDVMERADPLFLLIGLPTIPVMLILGKMIRWEDYVLRLWRKYSNKLQILNSIFPGIGCPVPRIPAEANPLADHVSATRILCGALVFPTIATIVGKLMFSSVNSNLQRTILGGIAFVAIKGAFKVYFKQQQYLRQAHRKILNYPEQEEA",
# ELOV2_HUMAN
"MEHLKAFDDEINAFLDNMFGPRDSRVRGWFMLDSYLPTFFLTVMYLLSIWLGNKYMKNRPALSLRGILTLYNLGITLLSAYMLAELILSTWEGGYNLQCQDLTSAGEADIRVAKVLWWYYFSKSVEFLDTIFFVLRKKTSQITFLHVYHHASMFNIWWCVLNWIPCGQSFFGPTLNSFIHILMYSYYGLSVFPSMHKYLWWKKYLTQAQLVQFVLTITHTMSAVVKPCGFPFGCLIFQSSYMLTLVILFLNFYVQTYRKKPMKKDMQEPPAGKEVKNGFSKAYFTAANGVMNKKAQ",
# NSMA3_HUMAN
"MTTFGAVAEWRLPSLRRATLWIPQWFAKKAIFNSPLEAAMAFPHLQQPSFLLASLKADSINKPFAQQCQDLVKVIEDFPAKELHTIFPWLVESIFGSLDGVLVGWNLRCLQGRVNPVEYSIVMEFLDPGGPMMKLVYKLQAEDYKFDFPVSYLPGPVKASIQECILPDSPLYHNKVQFTPTGGLGLNLALNPFEYYIFFFALSLITQKPLPVSLHVRTSDCAYFILVDRYLSWFLPTEGSVPPPLSSSPGGTSPSPPPRTPAIPFASYGLHHTSLLKRHISHQTSVNADPASHEIWRSETLLQVFVEMWLHHYSLEMYQKMQSPHAKLEVLHYRLSVSSALYSPAQPSLQALHAYQESFTPTEEHVLVVRLLLKHLHAFANSLKPEQASPSAHSHATSPLEEFKRAAVPRFVQQKLYLFLQHCFGHWPLDASFRAVLEMWLSYLQPWRYAPDKQAPGSDSQPRCVSEKWAPFVQENLLMYTKLFVGFLNRALRTDLVSPKHALMVFRVAKVFAQPNLAEMIQKGEQLFLEPELVIPHRQHRLFTAPTFTGSFLSPWPPAVTDASFKVKSHVYSLEGQDCKYTPMFGPEARTLVLRLAQLITQAKHTAKSISDQCAESPAGHSFLSWLGFSSMDTNGSYTANDLDEMGQDSVRKTDEYLEKALEYLRQIFRLSEAQLRQFTLALGTTQDENGKKQLPDCIVGEDGLILTPLGRYQIINGLRRFEIEYQGDPELQPIRSYEIASLVRTLFRLSSAINHRFAGQMAALCSRDDFLGSFCRYHLTEPGLASRHLLSPVGRRQVAGHTRGPRLSLRFLGSYRTLVSLLLAFFVASLFCVGPLPCTLLLTLGYVLYASAMTLLTERGKLHQP",
# DJB12_HUMAN
"MESNKDEAERCISIALKAIQSNQPDRALRFLEKAQRLYPTPRVRALIESLNQKPQTAGDQPPPTDTTHATHRKAGGTDAPSANGEAGGESTKGYTAEQVAAVKRVKQCKDYYEILGVSRGASDEDLKKAYRRLALKFHPDKNHAPGATEAFKAIGTAYAVLSNPEKRKQYDQFGDDKSQAARHGHGHGDFHRGFEADISPEDLFNMFFGGGFPSSNVHVYSNGRMRYTYQQRQDRRDNQGDGGLGVFVQLMPILILILVSALSQLMVSSPPYSLSPRPSVGHIHRRVTDHLGVVYYVGDTFSEEYTGSSLKTVERNVEDDYIANLRNNCWKEKQQKEGLLYRARYFGDTDMYHRAQKMGTPSCSRLSEVQASLHG",
# TLR7_HUMAN
"MVFPMWTLKRQILILFNIILISKLLGARWFPKTLPCDVTLDVPKNHVIVDCTDKHLTEIPGGIPTNTTNLTLTINHIPDISPASFHRLDHLVEIDFRCNCVPIPLGSKNNMCIKRLQIKPRSFSGLTYLKSLYLDGNQLLEIPQGLPPSLQLLSLEANNIFSIRKENLTELANIEILYLGQNCYYRNPCYVSYSIEKDAFLNLTKLKVLSLKDNNVTAVPTVLPSTLTELYLYNNMIAKIQEDDFNNLNQLQILDLSGNCPRCYNAPFPCAPCKNNSPLQIPVNAFDALTELKVLRLHSNSLQHVPPRWFKNINKLQELDLSQNFLAKEIGDAKFLHFLPSLIQLDLSFNFELQVYRASMNLSQAFSSLKSLKILRIRGYVFKELKSFNLSPLHNLQNLEVLDLGTNFIKIANLSMFKQFKRLKVIDLSVNKISPSGDSSEVGFCSNARTSVESYEPQVLEQLHYFRYDKYARSCRFKNKEASFMSVNESCYKYGQTLDLSKNSIFFVKSSDFQHLSFLKCLNLSGNLISQTLNGSEFQPLAELRYLDFSNNRLDLLHSTAFEELHKLEVLDISSNSHYFQSEGITHMLNFTKNLKVLQKLMMNDNDISSSTSRTMESESLRTLEFRGNHLDVLWREGDNRYLQLFKNLLKLEELDISKNSLSFLPSGVFDGMPPNLKNLSLAKNGLKSFSWKKLQCLKNLETLDLSHNQLTTVPERLSNCSRSLKNLILKNNQIRSLTKYFLQDAFQLRYLDLSSNKIQMIQKTSFPENVLNNLKMLLLHHNRFLCTCDAVWFVWWVNHTEVTIPYLATDVTCVGPGAHKGQSVISLDLYTCELDLTNLILFSLSISVSLFLMVMMTASHLYFWDVWYIYHFCKAKIKGYQRLISPDCCYDAFIVYDTKDPAVTEWVLAELVAKLEDPREKHFNLCLEERDWLPGQPVLENLSQSIQLSKKTVFVMTDKYAKTENFKIAFYLSHQRLMDEKVDVIILIFLEKPFQKSKFLQLRKRLCGSSVLEWPTNPQAHPYFWQCLKNALATDNHVAYSQVFKETV",
# ELOV5_HUMAN
"MEHFDASLSTYFKALLGPRDTRVKGWFLLDNYIPTFICSVIYLLIVWLGPKYMRNKQPFSCRGILVVYNLGLTLLSLYMFCELVTGVWEGKYNFFCQGTRTAGESDMKIIRVLWWYYFSKLIEFMDTFFFILRKNNHQITVLHVYHHASMLNIWWFVMNWVPCGHSYFGATLNSFIHVLMYSYYGLSSVPSMRPYLWWKKYITQGQLLQFVLTIIQTSCGVIWPCTFPLGWLYFQIGYMISLIALFTNFYIQTYNKKGASRRKDHLKDHQNGSMAAVNGHTNSFSPLENNVKPRKLRKD",
# TECR_HUMAN
"MKHYEVEILDAKTREKLCFLDKVEPHATIAEIKNLFTKTHPQWYPARQSLRLDPKGKSLKDEDVLQKLPVGTTATLYFRDLGAQISWVTVFLTEYAGPLFIYLLFYFRVPFIYGHKYDFTSSRHTVVHLACICHSFHYIKRLLETLFVHRFSHGTMPLRNIFKNCTYYWGFAAWMAYYINHPLYTPPTYGAQQVKLALAIFVICQLGNFSIHMALRDLRPAGSKTRKIPYPTKNPFTWLFLLVSCPNYTYEVGSWIGFAIMTQCLPVALFSLVGFTQMTIWAKGKHRSYLKEFRDYPPLRMPIIPFLL",
# ERAP1_HUMAN
"MVFLPLKWSLATMSFLLSSLLALLTVSTPSWCQSTEASPKRSDGTPFPWNKIRLPEYVIPVHYDLLIHANLTTLTFWGTTKVEITASQPTSTIILHSHHLQISRATLRKGAGERLSEEPLQVLEHPRQEQIALLAPEPLLVGLPYTVVIHYAGNLSETFHGFYKSTYRTKEGELRILASTQFEPTAARMAFPCFDEPAFKASFSIKIRREPRHLAISNMPLVKSVTVAEGLIEDHFDVTVKMSTYLVAFIISDFESVSKITKSGVKVSVYAVPDKINQADYALDAAVTLLEFYEDYFSIPYPLPKQDLAAIPDFQSGAMENWGLTTYRESALLFDAEKSSASSKLGITMTVAHELAHQWFGNLVTMEWWNDLWLNEGFAKFMEFVSVSVTHPELKVGDYFFGKCFDAMEVDALNSSHPVSTPVENPAQIREMFDDVSYDKGACILNMLREYLSADAFKSGIVQYLQKHSYKNTKNEDLWDSMASICPTDGVKGMDGFCSRSQHSSSSSHWHQEGVDVKTMMNTWTLQKGFPLITITVRGRNVHMKQEHYMKGSDGAPDTGYLWHVPLTFITSKSDMVHRFLLKTKTDVLILPEEVEWIKFNVGMNGYYIVHYEDDGWDSLTGLLKGTHTAVSSNDRASLINNAFQLVSIGKLSIEKALDLSLYLKHETEIMPVFQGLNELIPMYKLMEKRDMNEVETQFKAFLIRLLRDLIDKQTWTDEGSVSERMLRSQLLLLACVHNYQPCVQRAEGYFRKWKESNGNLSLPVDVTLAVFAVGAQSTEGWDFLYSKYQFSLSSTEKSQIEFALCRTQNKEKLQWLLDESFKGDKIKTQEFPQILTLIGRNPVGYPLAWQFLRKNWNKLVQKFELGSSSIAHMVMGTTNQFSTRTRLEEVKGFFSSLKENGSQLRCVQQTIETIEENIGWMDKNFDKIRVWLQSEKLERM",
# PEN2_HUMAN
"MNLERVSNEEKLNLCRKYYLGGFAFLPFLWLVNIFWFFREAFLVPAYTEQSQIKGYVWRSAVGFLFWVIVLTSWITIFQIYRPRWGALGDYLSFTIPLGTP",
# USE1_HUMAN
"MAASRLELNLVRLLSRCEAMAAEKRDPDEWRLEKYVGALEDMLQALKVHASKPASEVINEYSWKVDFLKGMLQAEKLTSSSEKALANQFLAPGRVPTTARERVPATKTVHLQSRARYTSEMRSELLGTDSAEPEMDVRKRTGVAGSQPVSEKQLAAELDLVLQRHQNLQEKLAEEMLGLARSLKTNTLAAQSVIKKDNQTLSHSLKMADQNLEKLKTESERLEQHTQKSVNWLLWAMLIIVCFIFISMILFIRIMPKLK",
# E2AK3_HUMAN
"MERAISPGLLVRALLLLLLLLGLAARTVAAGRARGLPAPTAEAAFGLGAAAAPTSATRVPAAGAVAAAEVTVEDAEALPAAAGEQEPRGPEPDDETELRPRGRSLVIISTLDGRIAALDPENHGKKQWDLDVGSGSLVSSSLSKPEVFGNKMIIPSLDGALFQWDQDRESMETVPFTVESLLESSYKFGDDVVLVGGKSLTTYGLSAYSGKVRYICSALGCRQWDSDEMEQEEDILLLQRTQKTVRAVGPRSGNEKWNFSVGHFELRYIPDMETRAGFIESTFKPNENTEESKIISDVEEQEAAIMDIVIKVSVADWKVMAFSKKGGHLEWEYQFCTPIASAWLLKDGKVIPISLFDDTSYTSNDDVLEDEEDIVEAARGATENSVYLGMYRGQLYLQSSVRISEKFPSSPKALESVTNENAIIPLPTIKWKPLIHSPSRTPVLVGSDEFDKCLSNDKFSHEEYSNGALSILQYPYDNGYYLPYYKRERNKRSTQITVRFLDNPHYNKNIRKKDPVLLLHWWKEIVATILFCIIATTFIVRRLFHPHPHRQRKESETQCQTENKYDSVSGEANDSSWNDIKNSGYISRYLTDFEPIQCLGRGGFGVVFEAKNKVDDCNYAIKRIRLPNRELAREKVMREVKALAKLEHPGIVRYFNAWLEAPPEKWQEKMDEIWLKDESTDWPLSSPSPMDAPSVKIRRMDPFATKEHIEIIAPSPQRSRSFSVGISCDQTSSSESQFSPLEFSGMDHEDISESVDAAYNLQDSCLTDCDVEDGTMDGNDEGHSFELCPSEASPYVRSRERTSSSIVFEDSGCDNASSKEEPKTNRLHIGNHCANKLTAFKPTSSKSSSEATLSISPPRPTTLSLDLTKNTTEKLQPSSPKVYLYIQMQLCRKENLKDWMNGRCTIEERERSVCLHIFLQIAEAVEFLHSKGLMHRDLKPSNIFFTMDDVVKVGDFGLVTAMDQDEEEQTVLTPMPAYARHTGQVGTKLYMSPEQIHGNSYSHKVDIFSLGLILFELLYPFSTQMERVRTLTDVRNLKFPPLFTQKYPCEYVMVQDMLSPSPMERPEAINIIENAVFEDLDFPGKTVLRQRSRSLSSSGTKHSRQSNNSHSPLPSN",
# BFAR_HUMAN
"MEEPQKSYVNTMDLERDEPLKSTGPQISVSEFSCHCCYDILVNPTTLNCGHSFCRHCLALWWASSKKTECPECREKWEGFPKVSILLRDAIEKLFPDAIRLRFEDIQQNNDIVQSLAAFQKYGNDQIPLAPNTGRANQQMGGGFFSGVLTALTGVAVVLLVYHWSSRESEHDLLVHKAVAKWTAEEVVLWLEQLGPWASLYRERFLSERVNGRLLLTLTEEEFSKTPYTIENSSHRRAILMELERVKALGVKPPQNLWEYKAVNPGRSLFLLYALKSSPRLSLLYLYLFDYTDTFLPFIHTICPLQEDSSGEDIVTKLLDLKEPTWKQWREFLVKYSFLPYQLIAEFAWDWLEVHYWTSRFLIINAMLLSVLELFSFWRIWSRSELKTVPQRMWSHFWKVSTQGLFVAMFWPLIPQFVCNCLFYWALYFNPIINIDLVVKELRRLETQVL",
# CNIH4_HUMAN
"MEAVVFVFSLLDCCALIFLSVYFIITLSDLECDYINARSCCSKLNKWVIPELIGHTIVTVLLLMSLHWFIFLLNLPVATWNIYRYIMVPSGNMGVFDPTEIHNRGQLKSHMKEAMIKLGFHLLCFFMYLYSMILALIND",
# HACD3_HUMAN
"MENQVLTPHVYWAQRHRELYLRVELSDVQNPAISITENVLHFKAQGHGAKGDNVYEFHLEFLDLVKPEPVYKLTQRQVNITVQKKVSQWWERLTKQEKRPLFLAPDFDRWLDESDAEMELRAKEEERLNKLRLESEGSPETLTNLRKGYLFMYNLVQFLGFSWIFVNLTVRFCILGKESFYDTFHTVADMMYFCQMLAVVETINAAIGVTTSPVLPSLIQLLGRNFILFIIFGTMEEMQNKAVVFFVFYLWSAIEIFRYSFYMLTCIDMDWKVLTWLRYTLWIPLYPLGCLAEAVSVIQSIPIFNETGRFSFTLPYPVKIKVRFSFFLQIYLIMIFLGLYINFRHLYKQRRRRYGQKKKKIH",
# VAPA_HUMAN
"MASASGAMAKHEQILVLDPPTDLKFKGPFTDVVTTNLKLRNPSDRKVCFKVKTTAPRRYCVRPNSGIIDPGSTVTVSVMLQPFDYDPNEKSKHKFMVQTIFAPPNTSDMEAVWKEAKPDELMDSKLRCVFEMPNENDKLNDMEPSKAVPLNASKQDGPMPKPHSVSLNDTETRKLMEECKRLQGEMMKLSEENRHLRDEGLRLRKVAHSDKPGSTSTASFRDNVTSPLPSLLVVIAAIFIGFFLGKFIL",
# MARH2_HUMAN
"MTTGDCCHLPGSLCDCSGSPAFSKVVEATGLGPPQYVAQVTSRDGRLLSTVIRALDTPSDGPFCRICHEGANGECLLSPCGCTGTLGAVHKSCLEKWLSSSNTSYCELCHTEFAVEKRPRPLTEWLKDPGPRTEKRTLCCDMVCFLFITPLAAISGWLCLRGAQDHLRLHSQLEAVGLIALTIALFTIYVLWTLVSFRYHCQLYSEWRKTNQKVRLKIREADSPEGPQHSPLAAGLLKKVAEETPV",
# ORML1_HUMAN
"MNVGVAHSEVNPNTRVMNSRGMWLTYALGVGLLHIVLLSIPFFSVPVAWTLTNIIHNLGMYVFLHAVKGTPFETPDQGKARLLTHWEQLDYGVQFTSSRKFFTISPIILYFLASFYTKYDPTHFILNTASLLSVLIPKMPQLHGVRIFGINKY",
# AT10D_HUMAN
"MTEALQWARYHWRRLIRGATRDDDSGPYNYSSLLACGRKSSQTPKLSGRHRIVVPHIQPFKDEYEKFSGAYVNNRIRTTKYTLLNFVPRNLFEQFHRAANLYFLFLVVLNWVPLVEAFQKEITMLPLVVVLTIIAIKDGLEDYRKYKIDKQINNLITKVYSRKEKKYIDRCWKDVTVGDFIRLSCNEVIPADMVLLFSTDPDGICHIETSGLDGESNLKQRQVVRGYAEQDSEVDPEKFSSRIECESPNNDLSRFRGFLEHSNKERVGLSKENLLLRGCTIRNTEAVVGIVVYAGHETKAMLNNSGPRYKRSKLERRANTDVLWCVMLLVIMCLTGAVGHGIWLSRYEKMHFFNVPEPDGHIISPLLAGFYMFWTMIILLQVLIPISLYVSIEIVKLGQIYFIQSDVDFYNEKMDSIVQCRALNIAEDLGQIQYLFSDKTGTLTENKMVFRRCSVAGFDYCHEENARRLESYQEAVSEDEDFIDTVSGSLSNMAKPRAPSCRTVHNGPLGNKPSNHLAGSSFTLGSGEGASEVPHSRQAAFSSPIETDVVPDTRLLDKFSQITPRLFMPLDETIQNPPMETLYIIDFFIALAICNTVVVSAPNQPRQKIRHPSLGGLPIKSLEEIKSLFQRWSVRRSSSPSLNSGKEPSSGVPNAFVSRLPLFSRMKPASPVEEEVSQVCESPQCSSSSACCTETEKQHGDAGLLNGKAESLPGQPLACNLCYEAESPDEAALVYAARAYQCTLRSRTPEQVMVDFAALGPLTFQLLHILPFDSVRKRMSVVVRHPLSNQVVVYTKGADSVIMELLSVASPDGASLEKQQMIVREKTQKHLDDYAKQGLRTLCIAKKVMSDTEYAEWLRNHFLAETSIDNREELLLESAMRLENKLTLLGATGIEDRLQEGVPESIEALHKAGIKIWMLTGDKQETAVNIAYACKLLEPDDKLFILNTQSKDACGMLMSTILKELQKKTQALPEQVSLSEDLLQPPVPRDSGLRAGLIITGKTLEFALQESLQKQFLELTSWCQAVVCCRATPLQKSEVVKLVRSHLQVMTLAIGDGANDVSMIQVADIGIGVSGQEGMQAVMASDFAVSQFKHLSKLLLVHGHWCYTRLSNMILYFFYKNVAYVNLLFWYQFFCGFSGTSMTDYWVLIFFNLLFTSAPPVIYGVLEKDVSAETLMQLPELYRSGQKSEAYLPHTFWITLLDAFYQSLVCFFVPYFTYQGSDTDIFAFGNPLNTAALFIVLLHLVIESKSLTWIHLLVIIGSILSYFLFAIVFGAMCVTCNPPSNPYWIMQEHMLDPVFYLVCILTTSIALLPRFVYRVLQGSLFPSPILRAKHFDRLTPEERTKALKKWRGAGKMNQVTSKYANQSAGKSGRRPMPGPSAVFAMKSASSCAIEQGNLSLCETALDQGYSETKAFEMAGPSKGKES",
# STIM2_HUMAN
"MLVLGLLVAGAADGCELVPRHLRGRRATGSAATAASSPAAAAGDSPALMTDPCMSLSPPCFTEEDRFSLEALQTIHKQMDDDKDGGIEVEESDEFIREDMKYKDATNKHSHLHREDKHITIEDLWKRWKTSEVHNWTLEDTLQWLIEFVELPQYEKNFRDNNVKGTTLPRIAVHEPSFMISQLKISDRSHRQKLQLKALDVVLFGPLTRPPHNWMKDFILTVSIVIGVGGCWFAYTQNKTSKEHVAKMMKDLESLQTAEQSLMDLQERLEKAQEENRNVAVEKQNLERKMMDEINYAKEEACRLRELREGAECELSRRQYAEQELEQVRMALKKAEKEFELRSSWSVPDALQKWLQLTHEVEVQYYNIKRQNAEMQLAIAKDEAEKIKKKRSTVFGTLHVAHSSSLDEVDHKILEAKKALSELTTCLRERLFRWQQIEKICGFQIAHNSGLPSLTSSLYSDHSWVVMPRVSIPPYPIAGGVDDLDEDTPPIVSQFPGTMAKPPGSLARSSSLCRSRRSIVPSSPQPQRAQLAPHAPHPSHPRHPHHPQHTPHSLPSPDPDILSVSSCPALYRNEEEEEAIYFSAEKQWEVPDTASECDSLNSSIGRKQSPPLSLEIYQTLSPRKISRDEVSLEDSSRGDSPVTVDVSWGSPDCVGLTETKSMIFSPASKVYNGILEKSCSMNQLSSGIPVPKPRHTSCSSAGNDSKPVQEAPSVARISSIPHDLCHNGEKSKKPSKIKSLFKKKSK",
# RRBP1_HUMAN
"MDIYDTQTLGVVVFGGFMVVSAIGIFLVSTFSMKETSYEEALANQRKEMAKTHHQKVEKKKKEKTVEKKGKTKKKEEKPNGKIPDHDPAPNVTVLLREPVRAPAVAVAPTPVQPPIIVAPVATVPAMPQEKLASSPKDKKKKEKKVAKVEPAVSSVVNSIQVLTSKAAILETAPKEVPMVVVPPVGAKGNTPATGTTQGKKAEGTQNQSKKAEGAPNQGRKAEGTPNQGKKTEGTPNQGKKAEGTPNQGKKAEGTPNQGKKAEGAQNQGKKVDTTPNQGKKVEGAPTQGRKAEGAQNQAKKVEGAQNQGKKAEGAQNQGKKGEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKVEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGKKVEGAQNQGKKAEGAQNQGKKAEGAQNQGKKAEGAQNQGQKGEGAQNQGKKTEGAQGKKAERSPNQGKKGEGAPIQGKKADSVANQGTKVEGITNQGKKAEGSPSEGKKAEGSPNQGKKADAAANQGKKTESASVQGRNTDVAQSPEAPKQEAPAKKKSGSKKKGEPGPPDADGPLYLPYKTLVSTVGSMVFNEGEAQRLIEILSEKAGIIQDTWHKATQKGDPVAILKRQLEEKEKLLATEQEDAAVAKSKLRELNKEMAAEKAKAAAGEAKVKKQLVAREQEITAVQARMQASYREHVKEVQQLQGKIRTLQEQLENGPNTQLARLQQENSILRDALNQATSQVESKQNAELAKLRQELSKVSKELVEKSEAVRQDEQQRKALEAKAAAFEKQVLQLQASHRESEEALQKRLDEVSRELCHTQSSHASLRADAEKAQEQQQQMAELHSKLQSSEAEVRSKCEELSGLHGQLQEARAENSQLTERIRSIEALLEAGQARDAQDVQASQAEADQQQTRLKELESQVSGLEKEAIELREAVEQQKVKNNDLREKNWKAMEALATAEQACKEKLLSLTQAKEESEKQLCLIEAQTMEALLALLPELSVLAQQNYTEWLQDLKEKGPTLLKHPPAPAEPSSDLASKLREAEETQSTLQAECDQYRSILAETEGMLRDLQKSVEEEEQVWRAKVGAAEEELQKSRVTVKHLEEIVEKLKGELESSDQVREHTSHLEAELEKHMAAASAECQNYAKEVAGLRQLLLESQSQLDAAKSEAQKQSDELALVRQQLSEMKSHVEDGDIAGAPASSPEAPPAEQDPVQLKTQLEWTEAILEDEQTQRQKLTAEFEEAQTSACRLQEELEKLRTAGPLESSETEEASQLKERLEKEKKLTSDLGRAATRLQELLKTTQEQLAREKDTVKKLQEQLEKAEDGSSSKEGTSV",
# KLH14_HUMAN
"MSRSGDRTSTFDPSHSDNLLHGLNLLWRKQLFCDVTLTAQGQQFHCHKAVLASCSQYFRSLFSSHPPLGGGVGGQDGLGAPKDQQQPPQQQPSQQQQPPPQEEPGTPSSSPDDKLLTSPRAINNLVLQGCSSIGLRLVLEYLYTANVTLSLDTVEEVLSVSKILHIPQVTKLCVQFLNDQISVQNYKQVCKIAALHGLEETKKLANKYLVEDVLLLNFEEMRALLDSLPPPVESELALFQMSVLWLEHDRETRMQYAPDLMKRLRFALIPAPELVERVQSVDFMRTDPVCQKLLLDAMNYHLMPFRQHCRQSLASRIRSNKKMLLLVGGLPPGPDRLPSNLVQYYDDEKKTWKILTIMPYNSAHHCVVEVENFLFVLGGEDQWNPNGKHSTNFVSRYDPRFNSWIQLPPMQERRASFYACRLDKHLYVIGGRNETGYLSSVECYNLETNEWRYVSSLPQPLAAHAGAVHNGKIYISGGVHNGEYVPWLYCYDPVMDVWARKQDMNTKRAIHTLAVMNDRLYAIGGNHLKGFSHLDVMLVECYDPKGDQWNILQTPILEGRSGPGCAVLDDSIYLVGGYSWSMGAYKSSTICYCPEKGTWTELEGDVAEPLAGPACVTVILPSCVPYNK",
# DISP3_HUMAN
"MDTEDDPLLQDVWLEEEQEEEEATGETFLGAQKPGPQPGAGGQCCWRHWPLASRPPASGFWSTLGWAFTNPCCAGLVLFLGCSIPMALSAFMFLYYPPLDIDISYNAFEIRNHEASQRFDALTLALKSQFGSWGRNRRDLADFTSETLQRLISEQLQQLHLGNRSRQASRAPRVIPAASLGGPGPYRDTSAAQKPTANRSGRLRRETPPLEDLAANQSEDPRNQRLSKNGRYQPSIPPHAAVAANQSRARRGASRWDYSRAYVSANTQTHAHWRIELIFLARGDAERNIFTSERLVTIHEIERKIMDHPGFREFCWKPHEVLKDLPLGSYSYCSPPSSLMTYFFPTERGGKIYYDGMGQDLADIRGSLELAMTHPEFYWYVDEGLSADNLKSSLLRSEILFGAPLPNYYSVDDRWEEQRAKFQSFVVTYVAMLAKQSTSKVQVLYGGTDLFDYEVRRTFNNDMLLAFISSSCIAALVYILTSCSVFLSFFGIASIGLSCLVALFLYHVVFGIQYLGILNGVAAFVIVGIGVDDVFVFINTYRQATHLEDPQLRMIHTVQTAGKATFFTSLTTAAAYAANVFSQIPAVHDFGLFMSLIVSCCWLAVLVTMPAALGLWSLYLAPLESSCQTSCHQNCSRKTSLHFPGDVFAAPEQVGGSPAQGPIPYLDDDIPLLEVEEEPVSLELGDVSLVSVSPEGLQPASNTGSRGHLIVQLQELLHHWVLWSAVKSRWVIVGLFVSILILSLVFASRLRPASRAPLLFRPDTNIQVLLDLKYNLSAEGISCITCSGLFQEKPHSLQNNIRTSLEKKRRGSGVPWASRPEATLQDFPGTVYISKVKSQGHPAVYRLSLNASLPAPWQAVSPGDGEVPSFQVYRAPFGNFTKKLTACMSTVGLLQAASPSRKWMLTTLACDAKRGWKFDFSFYVATKEQQHTRKLYFAQSHKPPFHGRVCMAPPGCLLSSSPDGPTKGFFFVPSEKVPKARLSATFGFNPCVNTGCGKPAVRPLVDTGAMVFVVFGIIGVNRTRQVDNHVIGDPGSVVYDSSFDLFKEIGHLCHLCKAIAANSELVKPGGAQCLPSGYSISSFLQMLHPECKELPEPNLLPGQLSHGAVGVREGRVQWISMAFESTTYKGKSSFQTYSDYLRWESFLQQQLQALPEGSVLRRGFQTCEHWKQIFMEIVGVQSALCGLVLSLLICVAAVAVFTTHILLLLPVLLSILGIVCLVVTIMYWSGWEMGAVEAISLSILVGSSVDYCVHLVEGYLLAGENLPPHQAEDARTQRQWRTLEAVRHVGVAIVSSALTTVIATVPLFFCIIAPFAKFGKIVALNTGVSILYTLTVSTALLGIMAPSSFTRTRTSFLKALGAVLLAGALGLGACLVLLQSGYKIPLPAGASL",
# STX18_HUMAN
"MAVDITLLFRASVKTVKTRNKALGVAVGGGVDGSRDELFRRSPRPKGDFSSRAREVISHIGKLRDFLLEHRKDYINAYSHTMSEYGRMTDTERDQIDQDAQIFMRTCSEAIQQLRTEAHKEIHSQQVKEHRTAVLDFIEDYLKRVCKLYSEQRAIRVKRVVDKKRLSKLEPEPNTKTRESTSSEKVSQSPSKDSEENPATEERPEKILAETQPELGTWGDGKGEDELSPEEIQMFEQENQRLIGEMNSLFDEVRQIEGRVVEISRLQEIFTEKVLQQEAEIDSIHQLVVGATENIKEGNEDIREAIKNNAGFRVWILFFLVMCSFSLLFLDWYDS",
# DPM3_HUMAN
"MTKLAQWLWGLAILGSTWVALTTGALGLELPLSCQEVLWPLPAYLLVSAGCYALGTVGYRVATFHDCEDAARELQSQIQEARADLARRGLRF",
# DHCR7_HUMAN
"MAAKSQPNIPKAKSLDGVTNDRTASQGQWGRAWEVDWFSLASVIFLLLFAPFIVYYFIMACDQYSCALTGPVVDIVTGHARLSDIWAKTPPITRKAAQLYTLWVTFQVLLYTSLPDFCHKFLPGYVGGIQEGAVTPAGVVNKYQINGLQAWLLTHLLWFANAHLLSWFSPTIIFDNWIPLLWCANILGYAVSTFAMVKGYFFPTSARDCKFTGNFFYNYMMGIEFNPRIGKWFDFKLFFNGRPGIVAWTLINLSFAAKQRELHSHVTNAMVLVNVLQAIYVIDFFWNETWYLKTIDICHDHFGWYLGWGDCVWLPYLYTLQGLYLVYHPVQLSTPHAVGVLLLGLVGYYIFRVANHQKDLFRRTDGRCLIWGRKPKVIECSYTSADGQRHHSKLLVSGFWGVARHFNYVGDLMGSLAYCLACGGGHLLPYFYIIYMAILLTHRCLRDEHRCASKYGRDWERYTAAVPYRLLPGIF",
# SPAST_HUMAN
"MNSPGGRGKKKGSGGASNPVPPRPPPPCLAPAPPAAGPAPPPESPHKRNLYYFSYPLFVGFALLRLVAFHLGLLFVWLCQRFSRALMAAKRSSGAAPAPASASAPAPVPGGEAERVRVFHKQAFEYISIALRIDEDEKAGQKEQAVEWYKKGIEELEKGIAVIVTGQGEQCERARRLQAKMMTNLVMAKDRLQLLEKMQPVLPFSKSQTDVYNDSTNLACRNGHLQSESGAVPKRKDPLTHTSNSLPRSKTVMKTGSAGLSGHHRAPSYSGLSMVSGVKQGSGPAPTTHKGTPKTNRTNKPSTPTTATRKKKDLKNFRNVDSNLANLIMNEIVDNGTAVKFDDIAGQDLAKQALQEIVILPSLRPELFTGLRAPARGLLLFGPPGNGKTMLAKAVAAESNATFFNISAASLTSKYVGEGEKLVRALFAVARELQPSIIFIDEVDSLLCERREGEHDASRRLKTEFLIEFDGVQSAGDDRVLVMGATNRPQELDEAVLRRFIKRVYVSLPNEETRLLLLKNLLCKQGSPLTQKELAQLARMTDGYSGSDLTALAKDAALGPIRELKPEQVKNMSASEMRNIRLSDFTESLKKIKRSVSPQTLEAYIRWNKDFGDTTV",
# EXTL2_HUMAN
"MRCCHICKLPGRVMGIRVLRLSLVVILVLLLVAGALTALLPSVKEDKMLMLRREIKSQGKSTMDSFTLIMQTYNRTDLLLKLLNHYQAVPNLHKVIVVWNNIGEKAPDELWNSLGPHPIPVIFKQQTANRMRNRLQVFPELETNAVLMVDDDTLISTPDLVFAFSVWQQFPDQIVGFVPRKHVSTSSGIYSYGSFEMQAPGSGNGDQYSMVLIGASFFNSKYLELFQRQPAAVHALIDDTQNCDDIAMNFIIAKHIGKTSGIFVKPVNMDNLEKETNSGYSGMWHRAEHALQRSYCINKLVNIYDSMPLRYSNIMISQFGFPYANYKRKI",
# DNJB9_HUMAN
"MATPQSIFIFAICILMITELILASKSYYDILGVPKSASERQIKKAFHKLAMKYHPDKNKSPDAEAKFREIAEAYETLSDANRRKEYDTLGHSAFTSGKGQRGSGSSFEQSFNFNFDDLFKDFGFFGQNQNTGSKKRFENHFQTRQDGGSSRQRHHFQEFSFGGGLFDDMFEDMEKMFSFSGFDSTNQHTVQTENRFHGSSKHCRTVTQRRGNMVTTYTDCSGQ",
# SE1L1_HUMAN
"MRVRIGLTLLLCAVLLSLASASSDEEGSQDESLDSKTTLTSDESVKDHTTAGRVVAGQIFLDSEESELESSIQEEEDSLKSQEGESVTEDISFLESPNPENKDYEEPKKVRKPALTAIEGTAHGEPCHFPFLFLDKEYDECTSDGREDGRLWCATTYDYKADEKWGFCETEEEAAKRRQMQEAEMMYQTGMKILNGSNKKSQKREAYRYLQKAASMNHTKALERVSYALLFGDYLPQNIQAAREMFEKLTEEGSPKGQTALGFLYASGLGVNSSQAKALVYYTFGALGGNLIAHMVLGYRYWAGIGVLQSCESALTHYRLVANHVASDISLTGGSVVQRIRLPDEVENPGMNSGMLEEDLIQYYQFLAEKGDVQAQVGLGQLHLHGGRGVEQNHQRAFDYFNLAANAGNSHAMAFLGKMYSEGSDIVPQSNETALHYFKKAADMGNPVGQSGLGMAYLYGRGVQVNYDLALKYFQKAAEQGWVDGQLQLGSMYYNGIGVKRDYKQALKYFNLASQGGHILAFYNLAQMHASGTGVMRSCHTAVELFKNVCERGRWSERLMTAYNSYKDGDYNAAVIQYLLLAEQGYEVAQSNAAFILDQREASIVGENETYPRALLHWNRAASQGYTVARIKLGDYHFYGFGTDVDYETAFIHYRLASEQQHSAQAMFNLGYMHEKGLGIKQDIHLAKRFYDMAAEASPDAQVPVFLALCKLGVVYFLQYIRETNIRDMFTQLDMDQLLGPEWDLYLMTIIALLLGTVIAYRQRQHQDMPAPRPPGPRPAPPQQEGPPEQQPPQ",
# CLN8_HUMAN
"MNPASDGGTSESIFDLDYASWGIRSTLMVAGFVFYLGVFVVCHQLSSSLNATYRSLVAREKVFWDLAATRAVFGVQSTAAGLWALLGDPVLHADKARGQQNWCWFHITTATGFFCFENVAVHLSNLIFRTFDLFLVIHHLFAFLGFLGCLVNLQAGHYLAMTTLLLEMSTPFTCVSWMLLKAGWSESLFWKLNQWLMIHMFHCRMVLTYHMWWVCFWHWDGLVSSLYLPHLTLFLVGLALLTLIINPYWTHKKTQQLLNPVDWNFAQPEAKSRPEGNGQLLRKKRP",
# SEC63_HUMAN
"MAGQQFQYDDSGNTFFYFLTSFVGLIVIPATYYLWPRDQNAEQIRLKNIRKVYGRCMWYRLRLLKPQPNIIPTVKKIVLLAGWALFLFLAYKVSKTDREYQEYNPYEVLNLDPGATVAEIKKQYRLLSLKYHPDKGGDEVMFMRIAKAYAALTDEESRKNWEEFGNPDGPQATSFGIALPAWIVDQKNSILVLLVYGLAFMVILPVVVGSWWYRSIRYSGDQILIRTTQIYTYFVYKTRNMDMKRLIMVLAGASEFDPQYNKDATSRPTDNILIPQLIREIGSINLKKNEPPLTCPYSLKARVLLLSHLARMKIPETLEEDQQFMLKKCPALLQEMVNVICQLIVMARNREEREFRAPTLASLENCMKLSQMAVQGLQQFKSPLLQLPHIEEDNLRRVSNHKKYKIKTIQDLVSLKESDRHTLLHFLEDEKYEEVMAVLGSFPYVTMDIKSQVLDDEDSNNITVGSLVTVLVKLTRQTMAEVFEKEQSICAAEEQPAEDGQGETNKNRTKGGWQQKSKGPKKTAKSKKKKPLKKKPTPVLLPQSKQQKQKQANGVVGNEAAVKEDEEEVSDKGSDSEEEETNRDSQSEKDDGSDRDSDREQDEKQNKDDEAEWQELQQSIQRKERALLETKSKITHPVYSLYFPEEKQEWWWLYIADRKEQTLISMPYHVCTLKDTEEVELKFPAPGKPGNYQYTVFLRSDSYMGLDQIKPLKLEVHEAKPVPENHPQWDTAIEGDEDQEDSEGFEDSFEEEEEEEEDDD",
# NAT8_HUMAN
"MAPCHIRKYQESDRQWVVGLLSRGMAEHAPATFRQLLKLPRTLILLLGGPLALLLVSGSWLLALVFSISLFPALWFLAKKPWTEYVDMTLCTDMSDITKSYLSERGSCFWVAESEEKVVGMVGALPVDDPTLREKRLQLFHLFVDSEHRRQGIAKALVRTVLQFARDQGYSEVILDTGTIQLSAMALYQSMGFKKTGQSFFCVWARLVALHTVHFIYHLPSSKVGSL",
# ZDHC2_HUMAN
"MAPSGPGSSARRRCRRVLYWIPVVFITLLLGWSYYAYAIQLCIVSMENTGEQVVCLMAYHLLFAMFVWSYWKTIFTLPMNPSKEFHLSYAEKDLLEREPRGEAHQEVLRRAAKDLPIYTRTMSGAIRYCDRCQLIKPDRCHHCSVCDKCILKMDHHCPWVNNCVGFSNYKFFLLFLAYSLLYCLFIAATDLQYFIKFWTNGLPDTQAKFHIMFLFFAAAMFSVSLSSLFGYHCWLVSKNKSTLEAFRSPVFRHGTDKNGFSLGFSKNMRQVFGDEKKYWLLPIFSSLGDGCSFPTCLVNQDPEQASTPAGLNSTAKNLENHQFPAKPLRESQSHLLTDSQSWTESSINPGKCKAGMSNPALTMENET",
# MA1B1_HUMAN
"MAACEGRRSGALGSSQSDFLTPPVGGAPWAVATTVVMYPPPPPPPHRDFISVTLSFGENYDNSKSWRRRSCWRKWKQLSRLQRNMILFLLAFLLFCGLLFYINLADHWKALAFRLEEEQKMRPEIAGLKPANPPVLPAPQKADTDPENLPEISSQKTQRHIQRGPPHLQIRPPSQDLKDGTQEEATKRQEAPVDPRPEGDPQRTVISWRGAVIEPEQGTELPSRRAEVPTKPPLPPARTQGTPVHLNYRQKGVIDVFLHAWKGYRKFAWGHDELKPVSRSFSEWFGLGLTLIDALDTMWILGLRKEFEEARKWVSKKLHFEKDVDVNLFESTIRILGGLLSAYHLSGDSLFLRKAEDFGNRLMPAFRTPSKIPYSDVNIGTGVAHPPRWTSDSTVAEVTSIQLEFRELSRLTGDKKFQEAVEKVTQHIHGLSGKKDGLVPMFINTHSGLFTHLGVFTLGARADSYYEYLLKQWIQGGKQETQLLEDYVEAIEGVRTHLLRHSEPSKLTFVGELAHGRFSAKMDHLVCFLPGTLALGVYHGLPASHMELAQELMETCYQMNRQMETGLSPEIVHFNLYPQPGRRDVEVKPADRHNLLRPETVESLFYLYRVTGDRKYQDWGWEILQSFSRFTRVPSGGYSSINNVQDPQKPEPRDKMESFFLGETLKYLFLLFSDDPNLLSLDAYVFNTEAHPLPIWTPA",
# AMFR_HUMAN
"MPLLFLERFPWPSLRTYTGLSGLALLGTIISAYRALSQPEAGPGEPDQLTASLQPEPPAPARPSAGGPRARDVAQYLLSDSLFVWVLVNTACCVLMLVAKLIQCIVFGPLRVSERQHLKDKFWNFIFYKFIFIFGVLNVQTVEEVVMWCLWFAGLVFLHLMVQLCKDRFEYLSFSPTTPMSSHGRVLSLLVAMLLSCCGLAAVCSITGYTHGMHTLAFMAAESLLVTVRTAHVILRYVIHLWDLNHEGTWEGKGTYVYYTDFVMELTLLSLDLMHHIHMLLFGNIWLSMASLVIFMQLRYLFHEVQRRIRRHKNYLRVVGNMEARFAVATPEELAVNNDDCAICWDSMQAARKLPCGHLFHNSCLRSWLEQDTSCPTCRMSLNIADNNRVREEHQGENLDENLVPVAAAEGRPRLNQHNHFFHFDGSRIASWLPSFSVEVMHTTNILGITQASNSQLNAMAHQIQEMFPQVPYHLVLQDLQLTRSVEITTDNILEGRIQVPFPTQRSDSIRPALNSPVERPSSDQEEGETSAQTERVPLDLSPRLEETLDFGEVEVEPSEVEDFEARGSRFSKSADERQRMLVQRKDELLQQARKRFLNKSSEDDAASESFLPSEGASSDPVTLRRRMLAAAAERRLQKQQTS",
# DSE_HUMAN
"MRTHTRGAPSVFFIYLLCFVSAYITDENPEVMIPFTNANYDSHPMLYFSRAEVAELQLRAASSHEHIAARLTEAVHTMLSSPLEYLPPWDPKDYSARWNEIFGNNLGALAMFCVLYPENIEARDMAKDYMERMAAQPSWLVKDAPWDEVPLAHSLVGFATAYDFLYNYLSKTQQEKFLEVIANASGYMYETSYRRGWGFQYLHNHQPTNCMALLTGSLVLMNQGYLQEAYLWTKQVLTIMEKSLVLLREVTDGSLYEGVAYGSYTTRSLFQYMFLVQRHFNINHFGHPWLKQHFAFMYRTILPGFQRTVAIADSNYNWFYGPESQLVFLDKFVMRNGSGNWLADQIRRNRVVEGPGTPSKGQRWCTLHTEFLWYDGSLKSVPPPDFGTPTLHYFEDWGVVTYGSALPAEINRSFLSFKSGKLGGRAIYDIVHRNKYKDWIKGWRNFNAGHEHPDQNSFTFAPNGVPFITEALYGPKYTFFNNVLMFSPAVSKSCFSPWVGQVTEDCSSKWSKYKHDLAASCQGRVVAAEEKNGVVFIRGEGVGAYNPQLNLKNVQRNLILLHPQLLLLVDQIHLGEESPLETAASFFHNVDVPFEETVVDGVHGAFIRQRDGLYKMYWMDDTGYSEKATFASVTYPRGYPYNGTNYVNVTMHLRSPITRAAYLFIGPSIDVQSFTVHGDSQQLDVFIATSKHAYATYLWTGEATGQSAFAQVIADRHKILFDRNSAIKSSIVPEVKDYAAIVEQNLQHFKPVFQLLEKQILSRVRNTASFRKTAERLLRFSDKRQTEEAIDRIFAISQQQQQQSKSKKNRRAGKRYKFVDAVPDIFAQIEVNEKKIRQKAQILAQKELPIDEDEEMKDLLDFADVTYEKHKNGGLIKGRFGQARMVTTTHSRAPSLSASYTRLFLILNIAIFFVMLAMQLTYFQRAQSLHGQRCLYAVLLIDSCILLWLYSSCSQSQC",
# TMCO1_HUMAN
"MPRKRKCDLRAVRVGLLLGGGGVYGSRFRFTFPGCRALSPWRVRVQRRRCEMSTMFADTLLIVFISVCTALLAEGITWVLVYRTDKYKRLKAEVEKQSKKLEKKKETITESAGRQQKKKIERQEEKLKNNNRDLSMVRMKSMFAIGFCFTALMGMFNSIFDGRVVAKLPFTPLSYIQGLSHRNLLGDDTTDCSFIFLYILCTMSIRQNIQKILGLAPSRAATKQAGGFLGPPPPSGKFS",
# PHTF1_HUMAN
"MASNERDAISWYQKKIGAYDQQIWEKSIEQTQIKGLKNKPKKMGHIKPDLIDVDLIRGSTFAKAKPEIPWTSLTRKGLVRVVFFPLFSNWWIQVTSLRIFVWLLLLYFMQVIAIVLYLMMPIVNISEVLGPLCLMLLMGTVHCQIVSTQITRPSGNNGNRRRRKLRKTVNGDGSRENGNNSSDKVRGIETLESVPIIGGFWETIFGNRIKRVKLISNKGTETDNDPSCVHPIIKRRQCRPEIRMWQTREKAKFSDGEKCRREAFRRLGNGVSDDLSSEEDGEARTQMILLRRSVEGASSDNGCEVKNRKSILSRHLNSQVKKTTTRWCHIVRDSDSLAESEFESAAFSQGSRSGVSGGSRSLNMSRRDSESTRHDSETEDMLWDDLLHGPECRSSVTSDSEGAHVNTLHSGTKRDPKEDVFQQNHLFWLQNSSPSSDRVSAIIWEGNECKKMDMSVLEISGIIMSRVNAYQQGVGYQMLGNVVTIGLAFFPFLHRLFREKSLDQLKSISAEEILTLFCGAPPVTPIIVLSIINFFERLCLTWMFFFMMCVAERTYKQRFLFAKLFSHITSARKARKYEIPHFRLKKVENIKIWLSLRSYLKRRGPQRSVDVVVSSVFLLTLSIAFICCAQVLQGHKTFLNDAYNWEFLIWETALLLFLLRLASLGSETNKKYSNVSILLTEQINLYLKMEKKPNKKEQLTLVNNVLKLSTKLLKELDTPFRLYGLTMNPLIYNITRVVILSAVSGVISDLLGFNIRLWKIKS",
# BOK_HUMAN
"MEVLRRSSVFAAEIMDAFDRSPTDKELVAQAKALGREYVHARLLRAGLSWSAPERAAPVPGRLAEVCAVLLRLGDELEMIRPSVYRNVARQLHISLQSEPVVTDAFLAVAGHIFSAGITWGKVVSLYAVAAGLAVDCVRQAQPAMVHALVDCLGEFVRKTLATWLRRRGGWTDVLKCVVSTDPGLRSHWLVAALCSFGRFLKAAFFVLLPER",
# S27A5_HUMAN
"MGVRQQLALLLLLLLLLWGLGQPVWPVAVALTLRWLLGDPTCCVLLGLAMLARPWLGPWVPHGLSLAAAALALTLLPARLPPGLRWLPADVIFLAKILHLGLKIRGCLSRQPPDTFVDAFERRARAQPGRALLVWTGPGAGSVTFGELDARACQAAWALKAELGDPASLCAGEPTALLVLASQAVPALCMWLGLAKLGCPTAWINPHGRGMPLAHSVLSSGARVLVVDPDLRESLEEILPKLQAENIRCFYLSHTSPTPGVGALGAALDAAPSHPVPADLRAGITWRSPALFIYTSGTTGLPKPAILTHERVLQMSKMLSLSGATADDVVYTVLPLYHVMGLVVGILGCLDLGATCVLAPKFSTSCFWDDCRQHGVTVILYVGELLRYLCNIPQQPEDRTHTVRLAMGNGLRADVWETFQQRFGPIRIWEVYGSTEGNMGLVNYVGRCGALGKMSCLLRMLSPFELVQFDMEAAEPVRDNQGFCIPVGLGEPGLLLTKVVSQQPFVGYRGPRELSERKLVRNVRQSGDVYYNTGDVLAMDREGFLYFRDRLGDTFRWKGENVSTHEVEGVLSQVDFLQQVNVYGVCVPGCEGKVGMAAVQLAPGQTFDGEKLYQHVRAWLPAYATPHFIRIQDAMEVTSTFKLMKTRLVREGFNVGIVVDPLFVLDNRAQSFRPLTAEMYQAVCEGTWRL",
# TMM98_HUMAN
"METVVIVAIGVLATIFLASFAALVLVCRQRYCRPRDLLQRYDSKPIVDLIGAMETQSEPSELELDDVVITNPHIEAILENEDWIEDASGLMSHCIAILKICHTLTEKLVAMTMGSGAKMKTSASVSDIIVVAKRISPRVDDVVKSMYPPLDPKLLDARTTALLLSVSHLVLVTRNACHLTGGLDWIDQSLSAAEEHLEVLREAALASEPDKGLPGPEGFLQEQSAI",
# ZDHC9_HUMAN
"MSVMVVRKKVTRKWEKLPGRNTFCCDGRVMMARQKGIFYLTLFLILGTCTLFFAFECRYLAVQLSPAIPVFAAMLFLFSMATLLRTSFSDPGVIPRALPDEAAFIEMEIEATNGAVPQGQRPPPRIKNFQINNQIVKLKYCYTCKIFRPPRASHCSICDNCVERFDHHCPWVGNCVGKRNYRYFYLFILSLSLLTIYVFAFNIVYVALKSLKIGFLETLKETPGTVLEVLICFFTLWSVVGLTGFHTFLVALNQTTNEDIKGSWTGKNRVQNPYSHGNIVKNCCEVLCGPLPPSVLDRRGILPLEESGSRPPSTQETSSSLLPQSPAPTEHLNSNEMPEDSSTPEEMPPPEPPEPPQEAAEAEK",
# TMED7_HUMAN
"MPRPGSAQRWAAVAGRWGCRLLALLLLVPGPGGASEITFELPDNAKQCFYEDIAQGTKCTLEFQVITGGHYDVDCRLEDPDGKVLYKEMKKQYDSFTFTASKNGTYKFCFSNEFSTFTHKTVYFDFQVGEDPPLFPSENRVSALTQMESACVSIHEALKSVIDYQTHFRLREAQGRSRAEDLNTRVAYWSVGEALILLVVSIGQVFLLKSFFSDKRTTTTRVGS",
# TMED3_HUMAN
"MGSTVPRSASVLLLLLLLRRAEQPCGAELTFELPDNAKQCFHEEVEQGVKFSLDYQVITGGHYDVDCYVEDPQGNTIYRETKKQYDSFTYRAEVKGVYQFCFSNEFSTFSHKTVYFDFQVGDEPPILPDMGNRVTALTQMESACVTIHEALKTVIDSQTHYRLREAQDRARAEDLNSRVSYWSVGETIALFVVSFSQVLLLKSFFTEKRPISRAVHS",
# GRIP1_HUMAN
"MIAVSFKCRCQILRRLTKDESPYTKSASQTKPPDGALAVRRQSIPEEFKGSTVVELMKKEGTTLGLTVSGGIDKDGKPRVSNLRQGGIAARSDQLDVGDYIKAVNGINLAKFRHDEIISLLKNVGERVVLEVEYELPPVSVQGSSVIFRTVEVTLHKEGNTFGFVIRGGAHDDRNKSRPVVITCVRPGGPADREGTIKPGDRLLSVDGIRLLGTTHAEAMSILKQCGQEAALLIEYDVSVMDSVATASGPLLVEVAKTPGASLGVALTTSMCCNKQVIVIDKIKSASIADRCGALHVGDHILSIDGTSMEYCTLAEATQFLANTTDQVKLEILPHHQTRLALKGPDHVKIQRSDRQLTWDSWASNHSSLHTNHHYNTYHPDHCRVPALTFPKAPPPNSPPALVSSSFSPTSMSAYSLSSLNMGTLPRSLYSTSPRGTMMRRRLKKKDFKSSLSLASSTVGLAGQVVHTETTEVVLTADPVTGFGIQLQGSVFATETLSSPPLISYIEADSPAERCGVLQIGDRVMAINGIPTEDSTFEEASQLLRDSSITSKVTLEIEFDVAESVIPSSGTFHVKLPKKHNVELGITISSPSSRKPGDPLVISDIKKGSVAHRTGTLELGDKLLAIDNIRLDNCSMEDAVQILQQCEDLVKLKIRKDEDNSDEQESSGAIIYTVELKRYGGPLGITISGTEEPFDPIIISSLTKGGLAERTGAIHIGDRILAINSSSLKGKPLSEAIHLLQMAGETVTLKIKKQTDAQSASSPKKFPISSHLSDLGDVEEDSSPAQKPGKLSDMYPSTVPSVDSAVDSWDGSAIDTSYGTQGTSFQASGYNFNTYDWRSPKQRGSLSPVTKPRSQTYPDVGLSYEDWDRSTASGFAGAADSAETEQEENFWSQALEDLETCGQSGILRELEEKADRRVSLRNMTLLATIMSGSTMSLNHEAPTPRSQLGRQASFQERSSSRPHYSQTTRSNTLPSDVGRKSVTLRKMKQEIKEIMSPTPVELHKVTLYKDSDMEDFGFSVADGLLEKGVYVKNIRPAGPGDLGGLKPYDRLLQVNHVRTRDFDCCLVVPLIAESGNKLDLVISRNPLASQKSIDQQSLPGDWSEQNSAFFQQPSHGGNLETREPTNTL",
# TBL2_HUMAN
"MELSQMSELMGLSVLLGLLALMATAAVARGWLRAGEERSGRPACQKANGFPPDKSSGSKKQKQYQRIRKEKPQQHNFTHRLLAAALKSHSGNISCMDFSSNGKYLATCADDRTIRIWSTKDFLQREHRSMRANVELDHATLVRFSPDCRAFIVWLANGDTLRVFKMTKREDGGYTFTATPEDFPKKHKAPVIDIGIANTGKFIMTASSDTTVLIWSLKGQVLSTINTNQMNNTHAAVSPCGRFVASCGFTPDVKVWEVCFGKKGEFQEVVRAFELKGHSAAVHSFAFSNDSRRMASVSKDGTWKLWDTDVEYKKKQDPYLLKTGRFEEAAGAAPCRLALSPNAQVLALASGSSIHLYNTRRGEKEECFERVHGECIANLSFDITGRFLASCGDRAVRLFHNTPGHRAMVEEMQGHLKRASNESTRQRLQQQLTQAQETLKSLGALKK",
# IR3IP_HUMAN
"MAFTLYSLLQAALLCVNAIAVLHEERFLKNIGWGTDQGIGGFGEEPGIKSQLMNLIRSVRTVMRVPLIIVNSIAIVLLLLFG",
# PEX16_HUMAN
"MEKLRLLGLRYQEYVTRHPAATAQLETAVRGFSYLLAGRFADSHELSELVYSASNLLVLLNDGILRKELRKKLPVSLSQQKLLTWLSVLECVEVFMEMGAAKVWGEVGRWLVIALVQLAKAVLRMLLLLWFKAGLQTSPPIVPLDRETQAQPPDGDHSPGNHEQSYVGKRSNRVVRTLQNTPSLHSRHWGAPQQREGRQQQHHEELSATPTPLGLQETIAEFLYIARPLLHLLSLGLWGQRSWKPWLLAGVVDVTSLSLLSDRKGLTRRERRELRRRTILLLYYLLRSPFYDRFSEARILFLLQLLADHVPGVGLVTRPLMDYLPTWQKIYFYSWG",
# UBIA1_HUMAN
"MAASQVLGEKINILSGETVKAGDRDPLGNDCPEQDRLPQRSWRQKCASYVLALRPWSFSASLTPVALGSALAYRSHGVLDPRLLVGCAVAVLAVHGAGNLVNTYYDFSKGIDHKKSDDRTLVDRILEPQDVVRFGVFLYTLGCVCAACLYYLSPLKLEHLALIYFGGLSGSFLYTGGIGFKYVALGDLIILITFGPLAVMFAYAIQVGSLAIFPLVYAIPLALSTEAILHSNNTRDMESDREAGIVTLAILIGPTFSYILYNTLLFLPYLVFSILATHCTISLALPLLTIPMAFSLERQFRSQAFNKLPQRTAKLNLLLGLFYVFGIILAPAGSLPKI",
# POMT1_HUMAN
"MWGFLKRPVVVTADINLSLVALTGMGLLSRLWRLTYPRAVVFDEVYYGQYISFYMKQIFFLDDSGPPFGHMVLALGGYLGGFDGNFLWNRIGAEYSSNVPVWSLRLLPALAGALSVPMAYQIVLELHFSHCAAMGAALLMLIENALITQSRLMLLESVLIFFNLLAVLSYLKFFNCQKHSPFSLSWWFWLTLTGVACSCAVGIKYMGVFTYVLVLGVAAVHAWHLLGDQTLSNVGADVQCCMRPACMGQMQMSQGVCVFCHLLARAVALLVIPVVLYLLFFYVHLILVFRSGPHDQIMSSAFQASLEGGLARITQGQPLEVAFGSQVTLRNVFGKPVPCWLHSHQDTYPMIYENGRGSSHQQQVTCYPFKDVNNWWIVKDPRRHQLVVSSPPRPVRHGDMVQLVHGMTTRSLNTHDVAAPLSPHSQEVSCYIDYNISMPAQNLWRLEIVNRGSDTDVWKTILSEVRFVHVNTSAVLKLSGAHLPDWGYRQLEIVGEKLSRGYHGSTVWNVEEHRYGASQEQRERERELHSPAQVDVSRNLSFMARFSELQWRMLALRSDDSEHKYSSSPLEWVTLDTNIAYWLHPRTSAQIHLLGNIVIWVSGSLALAIYALLSLWYLLRRRRNVHDLPQDAWLRWVLAGALCAGGWAVNYLPFFLMEKTLFLYHYLPALTFQILLLPVVLQHISDHLCRSQLQRSIFSALVVAWYSSACHVSNTLRPLTYGDKSLSPHELKALRWKDSWDILIRKH",
# CP46A_HUMAN
"MSPGLLLLGSAVLLAFGLCCTFVHRARSRYEHIPGPPRPSFLLGHLPCFWKKDEVGGRVLQDVFLDWAKKYGPVVRVNVFHKTSVIVTSPESVKKFLMSTKYNKDSKMYRALQTVFGERLFGQGLVSECNYERWHKQRRVIDLAFSRSSLVSLMETFNEKAEQLVEILEAKADGQTPVSMQDMLTYTAMDILAKAAFGMETSMLLGAQKPLSQAVKLMLEGITASRNTLAKFLPGKRKQLREVRESIRFLRQVGRDWVQRRREALKRGEEVPADILTQILKAEEGAQDDEGLLDNFVTFFIAGHETSANHLAFTVMELSRQPEIVARLQAEVDEVIGSKRYLDFEDLGRLQYLSQVLKESLRLYPPAWGTFRLLEEETLIDGVRVPGNTPLLFSTYVMGRMDTYFEDPLTFNPDRFGPGAPKPRFTYFPFSLGHRSCIGQQFAQMEVKVVMAKLLQRLEFRLVPGQRFGLQEQATLKPLDPVLCTLRPRGWQPAPPPPPC",
# SELK_HUMAN
"MVYISNGQVLDSRSQSPWRLSLITDFFWGIAEFVVLFFKTLLQQDVKKRRSYGNSSDSRYDDGRGPPGNPPRRMGRINHLRGPSPPPMAGGUGR",
# SERP1_HUMAN
"MVAKQRIRMANEKHSKNITQRGNVAKTSRNAPEEKASVGPWLLALFIFVVCGSAIFQIIQSIRMGM",
# BNIP3_HUMAN
"MSQNGAPGMQEESLQGSWVELHFSNNGNGGSVPASVSIYNGDMEKILLDAQHESGRSSSKSSHCDSPPRSQTPQDTNRASETDTHSIGEKNSSQSEEDDIERRKEVESILKKNSDWIWDWSSRPENIPPKEFLFKHPKRTATLSMRNTSVMKKGGIFSAEFLKVFLPSLLLSHLLAIGLGIYIGRRLTTSTSTF",
# AG10B_HUMAN
"MAQLEGYCFSAALSCTFLVSCLLFSAFSRALREPYMDEIFHLPQAQRYCEGHFSLSQWDPMITTLPGLYLVSVGVVKPAIWIFAWSEHVVCSIGMLRFVNLLFSVGNFYLLYLLFHKVQPRNKAASSIQRVLSTLTLAVFPTLYFFNFLYYTEAGSMFFTLFAYLMCLYGNHKTSAFLGFCGFMFRQTNIIWAVFCAGNVIAQKLTEAWKTELQKKEDRLPPIKGPFAEFRKILQFLLAYSMSFKNLSMLFCLTWPYILLGFLFCAFVVVNGGIVIGDRSSHEACLHFPQLFYFFSFTLFFSFPHLLSPSKIKTFLSLVWKHGILFLVVTLVSVFLVWKFTYAHKYLLADNRHYTFYVWKRVFQRYAILKYLLVPAYIFAGWSIADSLKSKPIFWNLMFFICLFIVIVPQKLLEFRYFILPYVIYRLNITLPPTSRLVCELSCYAIVNFITFYIFLNKTFQWPNSQDIQRFMW",
# S27A3_HUMAN
"MAALLLLPLLLLLPLLLLKLHLWPQLRWLPADLAFAVRALCCKRALRARALAAAAADPEGPEGGCSLAWRLAELAQQRAAHTFLIHGSRRFSYSEAERESNRAARAFLRALGWDWGPDGGDSGEGSAGEGERAAPGAGDAAAGSGAEFAGGDGAARGGGAAAPLSPGATVALLLPAGPEFLWLWFGLAKAGLRTAFVPTALRRGPLLHCLRSCGARALVLAPEFLESLEPDLPALRAMGLHLWAAGPGTHPAGISDLLAEVSAEVDGPVPGYLSSPQSITDTCLYIFTSGTTGLPKAARISHLKILQCQGFYQLCGVHQEDVIYLALPLYHMSGSLLGIVGCMGIGATVVLKSKFSAGQFWEDCQQHRVTVFQYIGELCRYLVNQPPSKAERGHKVRLAVGSGLRPDTWERFVRRFGPLQVLETYGLTEGNVATINYTGQRGAVGRASWLYKHIFPFSLIRYDVTTGEPIRDPQGHCMATSPGEPGLLVAPVSQQSPFLGYAGGPELAQGKLLKDVFRPGDVFFNTGDLLVCDDQGFLRFHDRTGDTFRWKGENVATTEVAEVFEALDFLQEVNVYGVTVPGHEGRAGMAALVLRPPHALDLMQLYTHVSENLPPYARPRFLRLQESLATTETFKQQKVRMANEGFDPSTLSDPLYVLDQAVGAYLPLTTARYSALLAGNLRI",
# S27A1_HUMAN
"MRAPGAGAASVVSLALLWLLGLPWTWSAAAALGVYVGSGGWRFLRIVCKTARRDLFGLSVLIRVRLELRRHQRAGHTIPRIFQAVVQRQPERLALVDAGTGECWTFAQLDAYSNAVANLFRQLGFAPGDVVAIFLEGRPEFVGLWLGLAKAGMEAALLNVNLRREPLAFCLGTSGAKALIFGGEMVAAVAEVSGHLGKSLIKFCSGDLGPEGILPDTHLLDPLLKEASTAPLAQIPSKGMDDRLFYIYTSGTTGLPKAAIVVHSRYYRMAAFGHHAYRMQAADVLYDCLPLYHSAGNIIGVGQCLIYGLTVVLRKKFSASRFWDDCIKYNCTVVQYIGEICRYLLKQPVREAERRHRVRLAVGNGLRPAIWEEFTERFGVRQIGEFYGATECNCSIANMDGKVGSCGFNSRILPHVYPIRLVKVNEDTMELLRDAQGLCIPCQAGEPGLLVGQINQQDPLRRFDGYVSESATSKKIAHSVFSKGDSAYLSGDVLVMDELGYMYFRDRSGDTFRWRGENVSTTEVEGVLSRLLGQTDVAVYGVAVPGVEGKAGMAAVADPHSLLDPNAIYQELQKVLAPYARPIFLRLLPQVDTTGTFKIQKTRLQREGFDPRQTSDRLFFLDLKQGHYLPLNEAVYTRICSGAFAL",
# SMS1_HUMAN
"MKEVVYWSPKKVADWLLENAMPEYCEPLEHFTGQDLINLTQEDFKKPPLCRVSSDNGQRLLDMIETLKMEHHLEAHKNGHANGHLNIGVDIPTPDGSFSIKIKPNGMPNGYRKEMIKIPMPELERSQYPMEWGKTFLAFLYALSCFVLTTVMISVVHERVPPKEVQPPLPDTFFDHFNRVQWAFSICEINGMILVGLWLIQWLLLKYKSIISRRFFCIVGTLYLYRCITMYVTTLPVPGMHFNCSPKLFGDWEAQLRRIMKLIAGGGLSITGSHNMCGDYLYSGHTVMLTLTYLFIKEYSPRRLWWYHWICWLLSVVGIFCILLAHDHYTVDVVVAYYITTRLFWWYHTMANQQVLKEASQMNLLARVWWYRPFQYFEKNVQGIVPRSYHWPFPWPVVHLSRQVKYSRLVNDT",
# TMX4_HUMAN
"MAGGRCGPQLTALLAAWIAAVAATAGPEEAALPPEQSRVQPMTASNWTLVMEGEWMLKFYAPWCPSCQQTDSEWEAFAKNGEILQISVGKVDVIQEPGLSGRFFVTTLPAFFHAKDGIFRRYRGPGIFEDLQNYILEKKWQSVEPLTGWKSPASLTMSGMAGLFSISGKIWHLHNYFTVTLGIPAWCSYVFFVIATLVFGLFMGLVLVVISECFYVPLPRHLSERSEQNRRSEEAHRAEQLQDAEEEKDDSNEEENKDSLVDDEEEKEDLGDEDEAEEEEEEDNLAAGVDEERSEANDQGPPGEDGVTREEVEPEEAEEGISEQPCPADTEVVEDSLRQRKSQHADKGL",
# SAR1A_HUMAN
"MSFIFEWIYNGFSSVLQFLGLYKKSGKLVFLGLDNAGKTTLLHMLKDDRLGQHVPTLHPTSEELTIAGMTFTTFDLGGHEQARRVWKNYLPAINGIVFLVDCADHSRLVESKVELNALMTDETISNVPILILGNKIDRTDAISEEKLREIFGLYGQTTGKGNVTLKELNARPMEVFMCSVLKRQGYGEGFRWLSQYID",
# ZNT1_HUMAN
"MGCWGRNRGRLLCMLALTFMFMVLEVVVSRVTSSLAMLSDSFHMLSDVLALVVALVAERFARRTHATQKNTFGWIRAEVMGALVNAIFLTGLCFAILLEAIERFIEPHEMQQPLVVLGVGVAGLLVNVLGLCLFHHHSGFSQDSGHGHSHGGHGHGHGLPKGPRVKSTRPGSSDINVAPGEQGPDQEETNTLVANTSNSNGLKLDPADPENPRSGDTVEVQVNGNLVREPDHMELEEDRAGQLNMRGVFLHVLGDALGSVIVVVNALVFYFSWKGCSEGDFCVNPCFPDPCKAFVEIINSTHASVYEAGPCWVLYLDPTLCVVMVCILLYTTYPLLKESALILLQTVPKQIDIRNLIKELRNVEGVEEVHELHVWQLAGSRIIATAHIKCEDPTSYMEVAKTIKDVFHNHGIHATTIQPEFASVGSKSSVVPCELACRTQCALKQCCGTLPQAPSGKDAEKTPAVSISCLELSNNLEKKPRRTKAENIPAVVIEIKNMPNKQPESSL",
# EST2_HUMAN
"MRLHRLRARLSAVACGLLLLLVRGQGQDSASPIRTTHTGQVLGSLVHVKGANAGVQTFLGIPFAKPPLGPLRFAPPEPPESWSGVRDGTTHPAMCLQDLTAVESEFLSQFNMTFPSDSMSEDCLYLSIYTPAHSHEGSNLPVMVWIHGGALVFGMASLYDGSMLAALENVVVVIIQYRLGVLGFFSTGDKHATGNWGYLDQVAALRWVQQNIAHFGGNPDRVTIFGESAGGTSVSSLVVSPISQGLFHGAIMESGVALLPGLIASSADVISTVVANLSACDQVDSEALVGCLRGKSKEEILAINKPFKMIPGVVDGVFLPRHPQELLASADFQPVPSIVGVNNNEFGWLIPKVMRIYDTQKEMDREASQAALQKMLTLLMLPPTFGDLLREEYIGDNGDPQTLQAQFQEMMADSMFVIPALQVAHFQCSRAPVYFYEFQHQPSWLKNIRPPHMKADHGDELPFVFRSFFGGNYIKFTEEEEQLSRKMMKYWANFARNGNPNGEGLPHWPLFDQEEQYLQLNLQPAVGRALKAHRLQFWKKALPQKIQELEEPEERHTEL",
# TOR1B_HUMAN
"MLRAGWLRGAAALALLLAARVVAAFEPITVGLAIGAASAITGYLSYNDIYCRFAECCREERPLNASALKLDLEEKLFGQHLATEVIFKALTGFRNNKNPKKPLTLSLHGWAGTGKNFVSQIVAENLHPKGLKSNFVHLFVSTLHFPHEQKIKLYQDQLQKWIRGNVSACANSVFIFDEMDKLHPGIIDAIKPFLDYYEQVDGVSYRKAIFIFLSNAGGDLITKTALDFWRAGRKREDIQLKDLEPVLSVGVFNNKHSGLWHSGLIDKNLIDYFIPFLPLEYRHVKMCVRAEMRARGSAIDEDIVTRVAEEMTFFPRDEKIYSDKGCKTVQSRLDFH",
# P4HA2_HUMAN
"MKLWVSALLMAWFGVLSCVQAEFFTSIGHMTDLIYAEKELVQSLKEYILVEEAKLSKIKSWANKMEALTSKSAADAEGYLAHPVNAYKLVKRLNTDWPALEDLVLQDSAAGFIANLSVQRQFFPTDEDEIGAAKALMRLQDTYRLDPGTISRGELPGTKYQAMLSVDDCFGMGRSAYNEGDYYHTVLWMEQVLKQLDAGEEATTTKSQVLDYLSYAVFQLGDLHRALELTRRLLSLDPSHERAGGNLRYFEQLLEEEREKTLTNQTEAELATPEGIYERPVDYLPERDVYESLCRGEGVKLTPRRQKRLFCRYHHGNRAPQLLIAPFKEEDEWDSPHIVRYYDVMSDEEIERIKEIAKPKLARATVRDPKTGVLTVASYRVSKSSWLEEDDDPVVARVNRRMQHITGLTVKTAELLQVANYGVGGQYEPHFDFSRNDERDTFKHLGTGNRVATFLNYMSDVEAGGATVFPDLGAAIWPKKGTAVFWYNLLRSGEGDYRTRHAACPVLVGCKWVSNKWFHERGQEFLRPCGSTEVD",
# CRTAP_HUMAN
"MEPGRRGAAALLALLCVACALRAGRAQYERYSFRSFPRDELMPLESAYRHALDKYSGEHWAESVGYLEISLRLHRLLRDSEAFCHRNCSAAPQPEPAAGLASYPELRLFGGLLRRAHCLKRCKQGLPAFRQSQPSREVLADFQRREPYKFLQFAYFKANNLPKAIAAAHTFLLKHPDDEMMKRNMAYYKSLPGAEDYIKDLETKSYESLFIRAVRAYNGENWRTSITDMELALPDFFKAFYECLAACEGSREIKDFKDFYLSIADHYVEVLECKIQCEENLTPVIGGYPVEKFVATMYHYLQFAYYKLNDLKNAAPCAVSYLLFDQNDKVMQQNLVYYQYHRDTWGLSDEHFQPRPEAVQFFNVTTLQKELYDFAKENIMDDDEGEVVEYVDDLLELEETS",
# STC2_HUMAN
"MCAERLGQFMTLALVLATFDPARGTDATNPPEGPQDRSSQQKGRLSLQNTAEIQHCLVNAGDVGCGVFECFENNSCEIRGLHGICMTFLHNAGKFDAQGKSFIKDALKCKAHALRHRFGCISRKCPAIREMVSQLQRECYLKHDLCAAAQENTRVIVEMIHFKDLLLHEPYVDLVNLLLTCGEEVKEAITHSVQVQCEQNWGSLCSILSFCTSAIQKPPTAPPERQPQVDRTKLSRAHHGEAGHHLPEPSSRETGRGAKGERGSKSHPNAHARGRVGGLGAQGPSGSSEWEDEQSEYSDIRR",
# A1AT_HUMAN
"MPSSVSWGILLLAGLCCLVPVSLAEDPQGDAAQKTDTSHHDQDHPTFNKITPNLAEFAFSLYRQLAHQSNSTNIFFSPVSIATAFAMLSLGTKADTHDEILEGLNFNLTEIPEAQIHEGFQELLRTLNQPDSQLQLTTGNGLFLSEGLKLVDKFLEDVKKLYHSEAFTVNFGDTEEAKKQINDYVEKGTQGKIVDLVKELDRDTVFALVNYIFFKGKWERPFEVKDTEEEDFHVDQVTTVKVPMMKRLGMFNIQHCKKLSSWVLLMKYLGNATAIFFLPDEGKLQHLENELTHDIITKFLENEDRRSASLHLPKLSITGTYDLKSVLGQLGITKVFSNGADLSGVTEEAPLKLSKAVHKAVLTIDEKGTEAAGAMFLEAIPMSIPPEVKFNKPFVFLMIEQNTKSPLFMGKVVNPTQK",
# APOE_HUMAN
"MKVLWAALLVTFLAGCQAKVEQAVETEPEPELRQQTEWQSGQRWELALGRFWDYLRWVQTLSEQVQEELLSSQVTQELRALMDETMKELKAYKSELEEQLTPVAEETRARLSKELQAAQARLGADMEDVCGRLVQYRGEVQAMLGQSTEELRVRLASHLRKLRKRLLRDADDLQKRLAVYQAGAREGAERGLSAIRERLGPLVEQGRVRAATVGSLAGQPLQERAQAWGERLRARMEEMGSRTRDRLDEVKEQVAEVRAKLEEQAQQIRLQAEAFQARLKSWFEPLVEDMQRQWAGLVEKVQAAVGTSAAPVPSDNH",
# ALBU_HUMAN
"MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIARRHPYFYAPELLFFAKRYKAAFTECCQAADKAACLLPKLDELRDEGKASSAKQRLKCASLQKFGERAFKAWAVARLSQRFPKAEFAEVSKLVTDLTKVHTECCHGDLLECADDRADLAKYICENQDSISSKLKECCEKPLLEKSHCIAEVENDEMPADLPSLAADFVESKDVCKNYAEAKDVFLGMFLYEYARRHPDYSVVLLLRLAKTYETTLEKCCAAADPHECYAKVFDEFKPLVEEPQNLIKQNCELFEQLGEYKFQNALLVRYTKKVPQVSTPTLVEVSRNLGKVGSKCCKHPEAKRMPCAEDYLSVVLNQLCVLHEKTPVSDRVTKCCTESLVNRRPCFSALEVDETYVPKEFNAETFTFHADICTLSEKERQIKKQTALVELVKHKPKATKEQLKAVMDDFAAFVEKCCKADDKETCFAEEGKKLVAASQAALGL",
# VTNC_HUMAN
"MAPLRPLLILALLAWVALADQESCKGRCTEGFNVDKKCQCDELCSYYQSCCTDYTAECKPQVTRGDVFTMPEDEYTVYDDGEEKNNATVHEQVGGPSLTSDLQAQSKGNPEQTPVLKPEEEAPAPEVGASKPEGIDSRPETLHPGRPQPPAEEELCSGKPFDAFTDLKNGSLFAFRGQYCYELDEKAVRPGYPKLIRDVWGIEGPIDAAFTRINCQGKTYLFKGSQYWRFEDGVLDPDYPRNISDGFDGIPDNVDAALALPAHSYSGRERVYFFKGKQYWEYQFQHQPSQEECEGSSLSAVFEHFAMMQRDSWEDIFELLFWGRTSAGTRQPQFISRDWHGVPGQVDAAMAGRIYISGMAPRPSLAKKQRFRHRNRKGYRSQRGHSRGRNQNSRRPSRATWLSLFSSEESNLGANNYDDYRMDWLVPATCEPIQSVFFFSGDKYYRVNLRTRRVDTVDPPYPRSIAQYWLGCPAPGHL",
# PROC_HUMAN
"MWQLTSLLLFVATWGISGTPAPLDSVFSSSERAHQVLRIRKRANSFLEELRHSSLERECIEEICDFEEAKEIFQNVDDTLAFWSKHVDGDQCLVLPLEHPCASLCCGHGTCIDGIGSFSCDCRSGWEGRFCQREVSFLNCSLDNGGCTHYCLEEVGWRRCSCAPGYKLGDDLLQCHPAVKFPCGRPWKRMEKKRSHLKRDTEDQEDQVDPRLIDGKMTRRGDSPWQVVLLDSKKKLACGAVLIHPSWVLTAAHCMDESKKLLVRLGEYDLRRWEKWELDLDIKEVFVHPNYSKSTTDNDIALLHLAQPATLSQTIVPICLPDSGLAERELNQAGQETLVTGWGYHSSREKEAKRNRTFVLNFIKIPVVPHNECSEVMSNMVSENMLCAGILGDRQDACEGDSGGPMVASFHGTWFLVGLVSWGEGCGLLHNYGVYTKVSRYLDWIHGHIRDKEAPQKSWAP",
# PDIA1_HUMAN
"MLRRALLCLAVAALVRADAPEEEDHVLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLKAEGSEIRLAKVDATEESDLAQQYGVRGYPTIKFFRNGDTASPKEYTAGREADDIVNWLKKRTGPAATTLPDGAAAESLVESSEVAVIGFFKDVESDSAKQFLQAAEAIDDIPFGITSNSDVFSKYQLDKDGVVLFKKFDEGRNNFEGEVTKENLLDFIKHNQLPLVIEFTEQTAPKIFGGEIKTHILLFLPKSVSDYDGKLSNFKTAAESFKGKILFIFIDSDHTDNQRILEFFGLKKEECPAVRLITLEEEMTKYKPESEELTAERITEFCHRFLEGKIKPHLMSQELPEDWDKQPVKVLVGKNFEDVAFDEKKNVFVEFYAPWCGHCKQLAPIWDKLGETYKDHENIVIAKMDSTANEVEAVKVHSFPTLKFFPASADRTVIDYNGERTLDGFKKFLESGGQDGAGDDDDLEDLEEAEEPDMEEDDDQKAVKDEL",
# TSP1_HUMAN
"MGLAWGLGVLFLMHVCGTNRIPESGGDNSVFDIFELTGAARKGSGRRLVKGPDPSSPAFRIEDANLIPPVPDDKFQDLVDAVRAEKGFLLLASLRQMKKTRGTLLALERKDHSGQVFSVVSNGKAGTLDLSLTVQGKQHVVSVEEALLATGQWKSITLFVQEDRAQLYIDCEKMENAELDVPIQSVFTRDLASIARLRIAKGGVNDNFQGVLQNVRFVFGTTPEDILRNKGCSSSTSVLLTLDNNVVNGSSPAIRTNYIGHKTKDLQAICGISCDELSSMVLELRGLRTIVTTLQDSIRKVTEENKELANELRRPPLCYHNGVQYRNNEEWTVDSCTECHCQNSVTICKKVSCPIMPCSNATVPDGECCPRCWPSDSADDGWSPWSEWTSCSTSCGNGIQQRGRSCDSLNNRCEGSSVQTRTCHIQECDKRFKQDGGWSHWSPWSSCSVTCGDGVITRIRLCNSPSPQMNGKPCEGEARETKACKKDACPINGGWGPWSPWDICSVTCGGGVQKRSRLCNNPTPQFGGKDCVGDVTENQICNKQDCPIDGCLSNPCFAGVKCTSYPDGSWKCGACPPGYSGNGIQCTDVDECKEVPDACFNHNGEHRCENTDPGYNCLPCPPRFTGSQPFGQGVEHATANKQVCKPRNPCTDGTHDCNKNAKCNYLGHYSDPMYRCECKPGYAGNGIICGEDTDLDGWPNENLVCVANATYHCKKDNCPNLPNSGQEDYDKDGIGDACDDDDDNDKIPDDRDNCPFHYNPAQYDYDRDDVGDRCDNCPYNHNPDQADTDNNGEGDACAADIDGDGILNERDNCQYVYNVDQRDTDMDGVGDQCDNCPLEHNPDQLDSDSDRIGDTCDNNQDIDEDGHQNNLDNCPYVPNANQADHDKDGKGDACDHDDDNDGIPDDKDNCRLVPNPDQKDSDGDGRGDACKDDFDHDSVPDIDDICPENVDISETDFRRFQMIPLDPKGTSQNDPNWVVRHQGKELVQTVNCDPGLAVGYDEFNAVDFSGTFFINTERDDDYAGFVFGYQSSSRFYVVMWKQVTQSYWDTNPTRAQGYSGLSVKVVNSTTGPGEHLRNALWHTGNTPGQVRTLWHDPRHIGWKDFTAYRWRLSHRPKTGFIRVVMYEGKKIMADSGPIYDKTYAGGRLGLFVFSQEMVFFSDLKYECRDP",
# CO1A2_HUMAN
"MLSFVDTRTLLLLAVTLCLATCQSLQEETVRKGPAGDRGPRGERGPPGPPGRDGEDGPTGPPGPPGPPGPPGLGGNFAAQYDGKGVGLGPGPMGLMGPRGPPGAAGAPGPQGFQGPAGEPGEPGQTGPAGARGPAGPPGKAGEDGHPGKPGRPGERGVVGPQGARGFPGTPGLPGFKGIRGHNGLDGLKGQPGAPGVKGEPGAPGENGTPGQTGARGLPGERGRVGAPGPAGARGSDGSVGPVGPAGPIGSAGPPGFPGAPGPKGEIGAVGNAGPAGPAGPRGEVGLPGLSGPVGPPGNPGANGLTGAKGAAGLPGVAGAPGLPGPRGIPGPVGAAGATGARGLVGEPGPAGSKGESGNKGEPGSAGPQGPPGPSGEEGKRGPNGEAGSAGPPGPPGLRGSPGSRGLPGADGRAGVMGPPGSRGASGPAGVRGPNGDAGRPGEPGLMGPRGLPGSPGNIGPAGKEGPVGLPGIDGRPGPIGPAGARGEPGNIGFPGPKGPTGDPGKNGDKGHAGLAGARGAPGPDGNNGAQGPPGPQGVQGGKGEQGPPGPPGFQGLPGPSGPAGEVGKPGERGLHGEFGLPGPAGPRGERGPPGESGAAGPTGPIGSRGPSGPPGPDGNKGEPGVVGAVGTAGPSGPSGLPGERGAAGIPGGKGEKGEPGLRGEIGNPGRDGARGAPGAVGAPGPAGATGDRGEAGAAGPAGPAGPRGSPGERGEVGPAGPNGFAGPAGAAGQPGAKGERGAKGPKGENGVVGPTGPVGAAGPAGPNGPPGPAGSRGDGGPPGMTGFPGAAGRTGPPGPSGISGPPGPPGPAGKEGLRGPRGDQGPVGRTGEVGAVGPPGFAGEKGPSGEAGTAGPPGTPGPQGLLGAPGILGLPGSRGERGLPGVAGAVGEPGPLGIAGPPGARGPPGAVGSPGVNGAPGEAGRDGNPGNDGPPGRDGQPGHKGERGYPGNIGPVGAAGAPGPHGPVGPAGKHGNRGETGPSGPVGPAGAVGPRGPSGPQGIRGDKGEPGEKGPRGLPGLKGHNGLQGLPGIAGHHGDQGAPGSVGPAGPRGPAGPSGPAGKDGRTGHPGTVGPAGIRGPQGHQGPAGPPGPPGPPGPPGVSGGGYDFGYDGDFYRADQPRSAPSLRPKDYEVDATLKSLNNQIETLLTPEGSRKNPARTCRDLRLSHPEWSSGYYWIDPNQGCTMDAIKVYCDFSTGETCIRAQPENIPAKNWYRSSKDKKHVWLGETINAGSQFEYNVEGVTSKEMATQLAFMRLLANYASQNITYHCKNSIAYMDEETGNLKKAVILQGSNDVELVAEGNSRFTYTVLVDGCSKKTNEWGKTIIEYKTNKPSRLPFLDIAPLDIGGADQEFFVDIGPVCFK",
# CLUS_HUMAN
"MMKTLLLFVGLLLTWESGQVLGDQTVSDNELQEMSNQGSKYVNKEIQNAVNGVKQIKTLIEKTNEERKTLLSNLEEAKKKKEDALNETRESETKLKELPGVCNETMMALWEECKPCLKQTCMKFYARVCRSGSGLVGRQLEEFLNQSSPFYFWMNGDRIDSLLENDRQQTHMLDVMQDHFSRASSIIDELFQDRFFTREPQDTYHYLPFSLPHRRPHFFFPKSRIVRSLMPFSPYEPLNFHAMFQPFLEMIHEAQQAMDIHFHSPAFQHPPTEFIREGDDDRTVCREIRHNSTGCLRMKDQCDKCREILSVDCSTNNPSQAKLRRELDESLQVAERLTRKYNELLKSYQWKMLNTSSLLEQLNEQFNWVSRLANLTQGEDQYYLRVTTVASHTSDSDVPSGVTEVVVKLFDSDPITVTVPVEVSRKNPKFMETVAEKALQEYRKKHREE",
# PDIA4_HUMAN
"MRPRKAFLLLLLLGLVQLLAVAGAEGPDEDSSNRENAIEDEEEEEEEDDDEEEDDLEVKEENGVLVLNDANFDNFVADKDTVLLEFYAPWCGHCKQFAPEYEKIANILKDKDPPIPVAKIDATSASVLASRFDVSGYPTIKILKKGQAVDYEGSRTQEEIVAKVREVSQPDWTPPPEVTLVLTKENFDEVVNDADIILVEFYAPWCGHCKKLAPEYEKAAKELSKRSPPIPLAKVDATAETDLAKRFDVSGYPTLKIFRKGRPYDYNGPREKYGIVDYMIEQSGPPSKEILTLKQVQEFLKDGDDVIIIGVFKGESDPAYQQYQDAANNLREDYKFHHTFSTEIAKFLKVSQGQLVVMQPEKFQSKYEPRSHMMDVQGSTQDSAIKDFVLKYALPLVGHRKVSNDAKRYTRRPLVVVYYSVDFSFDYRAATQFWRSKVLEVAKDFPEYTFAIADEEDYAGEVKDLGLSESGEDVNAAILDESGKKFAMEPEEFDSDTLREFVTAFKKGKLKPVIKSQPVPKNNKGPVKVVVGKTFDSIVMDPKKDVLIEFYAPWCGHCKQLEPVYNSLAKKYKGQKGLVIAKMDATANDVPSDRYKVEGFPTIYFAPSGDKKNPVKFEGGDRDLEHLSKFIEEHATKLSRTKEEL",
# P4HA1_HUMAN
"MIWYILIIGILLPQSLAHPGFFTSIGQMTDLIHTEKDLVTSLKDYIKAEEDKLEQIKKWAEKLDRLTSTATKDPEGFVGHPVNAFKLMKRLNTEWSELENLVLKDMSDGFISNLTIQRQYFPNDEDQVGAAKALLRLQDTYNLDTDTISKGNLPGVKHKSFLTAEDCFELGKVAYTEADYYHTELWMEQALRQLDEGEISTIDKVSVLDYLSYAVYQQGDLDKALLLTKKLLELDPEHQRANGNLKYFEYIMAKEKDVNKSASDDQSDQKTTPKKKGVAVDYLPERQKYEMLCRGEGIKMTPRRQKKLFCRYHDGNRNPKFILAPAKQEDEWDKPRIIRFHDIISDAEIEIVKDLAKPRLRRATISNPITGDLETVHYRISKSAWLSGYENPVVSRINMRIQDLTGLDVSTAEELQVANYGVGGQYEPHFDFARKDEPDAFKELGTGNRIATWLFYMSDVSAGGATVFPEVGASVWPKKGTAVFWYNLFASGEGDYSTRHAACPVLVGNKWVSNKWLHERGQEFRRPCTLSELE",
# GLU2B_HUMAN
"MLLPLLLLLPMCWAVEVKRPRGVSLTNHHFYDESKPFTCLDGSATIPFDQVNDDYCDCKDGSDEPGTAACPNGSFHCTNTGYKPLYIPSNRVNDGVCDCCDGTDEYNSGVICENTCKEKGRKERESLQQMAEVTREGFRLKKILIEDWKKAREEKQKKLIELQAGKKSLEDQVEMLRTVKEEAEKPEREAKEQHQKLWEEQLAAAKAQQEQELAADAFKELDDDMDGTVSVTELQTHPELDTDGDGALSEAEAQALLSGDTQTDATSFYDRVWAAIRDKYRSEALPTDLPAPSAPDLTEPKEEQPPVPSSPTEEEEEEEEEEEEEAEEEEEEEDSEEAPPPLSPPQPASPAEEDKMPPYDEQTQAFIDAAQEARNKFEEAERSLKDMEESIRNLEQEISFDFGPNGEFAYLYSQCYELTTNEYVYRLCPFKLVSQKPKLGGSPTSLGTWGSWIGPDHDKFSAMKYEQGTGCWQGPNRSTTVRLLCGKETMVTSTTEPSRCEYLMELMTPAACPEPPPEAPTEDDHDEL",
# EST1_HUMAN
"MWLRAFILATLSASAAWGHPSSPPVVDTVHGKVLGKFVSLEGFAQPVAIFLGIPFAKPPLGPLRFTPPQPAEPWSFVKNATSYPPMCTQDPKAGQLLSELFTNRKENIPLKLSEDCLYLNIYTPADLTKKNRLPVMVWIHGGGLMVGAASTYDGLALAAHENVVVVTIQYRLGIWGFFSTGDEHSRGNWGHLDQVAALRWVQDNIASFGGNPGSVTIFGESAGGESVSVLVLSPLAKNLFHRAISESGVALTSVLVKKGDVKPLAEQIAITAGCKTTTSAVMVHCLRQKTEEELLETTLKMKFLSLDLQGDPRESQPLLGTVIDGMLLLKTPEELQAERNFHTVPYMVGINKQEFGWLIPMQLMSYPLSEGQLDQKTAMSLLWKSYPLVCIAKELIPEATEKYLGGTDDTVKKKDLFLDLIADVMFGVPSVIVARNHRDAGAPTYMYEFQYRPSFSSDMKPKTVIGDHGDELFSVFGAPFLKEGASEEEIRLSKMVMKFWANFARNGNPNGEGLPHWPEYNQKEGYLQIGANTQAAQKLKDKEVAFWTNLFAKKAVEKPPQTEHIEL",
# PPIB_HUMAN
"MLRLSERNMKVLLAAALIAGSVFFLLLPGPSAADEKKKGPKVTVKVYFDLRIGDEDVGRVIFGLFGKTVPKTVDNFVALATGEKGFGYKNSKFHRVIKDFMIQGGDFTRGDGTGGKSIYGERFPDENFKLKHYGPGWVSMANAGKDTNGSQFFITTVKTAWLDGKHVVFGKVLEGMEVVRKVESTKTDSRDKPLKDVIIADCGKIEVEKPFAIAKE",
# ERP29_HUMAN
"MAAAVPRAAFLSPLLPLLLGFLLLSAPHGGSGLHTKGALPLDTVTFYKVIPKSKFVLVKFDTQYPYGEKQDEFKRLAENSASSDDLLVAEVGISDYGDKLNMELSEKYKLDKESYPVFYLFRDGDFENPVPYTGAVKVGAIQRWLKGQGVYLGMPGCLPVYDALAGEFIRASGVEARQALLKQGQDNLSSVKETQKKWAEQYLKIMGKILDQGEDFPASEMTRIARLIEKNKMSDGKKEELQKSLNILTAFQKKGAEKEEL",
# AMRP_HUMAN
"MAPRRVRSFLRGLPALLLLLLFLGPWPAASHGGKYSREKNQPKPSPKRESGEEFRMEKLNQLWEKAQRLHLPPVRLAELHADLKIQERDELAWKKLKLDGLDEDGEKEARLIRNLNVILAKYGLDGKKDARQVTSNSLSGTQEDGLDDPRLEKLWHKAKTSGKFSGEELDKLWREFLHHKEKVHEYNVLLETLSRTEEIHENVISPSDLSDIKGSVLHSRHTELKEKLRSINQGLDRLRRVSHQGYSTEAEFEEPRVIDLWDLAQSANLTDKELEAFREELKHFEAKIEKHNHYQKQLEIAHEKLRHAESVGDGERVSRSREKHALLEGRTKELGYTVKKHLQDLSGRISRARHNEL",
# FLT3_HUMAN
"MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSSVGKSSSYPMVSESPEDLGCALRPQSSGTVYEAAAVEVDVSASITLQVLVDAPGNISCLWVFKHSSLNCQPHFDLQNRGVVSMVILKMTETQAGEYLLFIQSEATNYTILFTVSIRNTLLYTLRRPYFRKMENQDALVCISESVPEPIVEWVLCDSQGESCKEESPAVVKKEEKVLHELFGTDIRCCARNELGRECTRLFTIDLNQTPQTTLPQLFLKVGEPLWIRCKAVHVNHGFGLTWELENKALEEGNYFEMSTYSTNRTMIRILFAFVSSVARNDTGYYTCSSSKHPSQSALVTIVEKGFINATNSSEDYEIDQYEEFCFSVRFKAYPQIRCTWTFSRKSFPCEQKGLDNGYSISKFCNHKHQPGEYIFHAENDDAQFTKMFTLNIRRKPQVLAEASASQASCFSDGYPLPSWTWKKCSDKSPNCTEEITEGVWNRKANRKVFGQWVSSSTLNMSEAIKGFLVKCCAYNSLGTSCETILLNSPGPFPFIQDNISFYATIGVCLLFIVVLTLLICHKYKKQFRYESQLQMVQVTGSSDNEYFYVDFREYEYDLKWEFPRENLEFGKVLGSGAFGKVMNATAYGISKTGVSIQVAVKMLKEKADSSEREALMSELKMMTQLGSHENIVNLLGACTLSGPIYLIFEYCCYGDLLNYLRSKREKFHRTWTEIFKEHNFSFYPTFQSHPNSSMPGSREVQIHPDSDQISGLHGNSFHSEDEIEYENQKRLEEEEDLNVLTFEDLLCFAYQVAKGMEFLEFKSCVHRDLAARNVLVTHGKVVKICDFGLARDIMSDSNYVVRGNARLPVKWMAPESLFEGIYTIKSDVWSYGILLWEIFSLGVNPYPGIPVDANFYKLIQNGFKMDQPFYATEEIYIIMQSCWAFDSRKRPSFPNLTSFLGCQLADAEEAMYQNVDGRVSECPHTYQNRRPFSREMDLGLLSPQAQVEDS",
# SERPH_HUMAN
"MRSLLLLSAFCLLEAALAAEVKKPAAAAAPGTAEKLSPKAATLAERSAGLAFSLYQAMAKDQAVENILVSPVVVASSLGLVSLGGKATTASQAKAVLSAEQLRDEEVHAGLGELLRSLSNSTARNVTWKLGSRLYGPSSVSFADDFVRSSKQHYNCEHSKINFRDKRSALQSINEWAAQTTDGKLPEVTKDVERTDGALLVNAMFFKPHWDEKFHHKMVDNRGFMVTRSYTVGVMMMHRTGLYNYYDDEKEKLQIVEMPLAHKLSSLIILMPHHVEPLERLEKLLTKEQLKIWMGKMQKKAVAISLPKGVVEVTHDLQKHLAGLGLTEAIDKNKADLSRMSGKKDLYLASVFHATAFELDTDGNPFDQDIYGREELRSPKLFYADHPFIFLVRDTQSGSLLFIGRLVRPKGDKMRDEL",
# MANF_HUMAN
"MRRMWATQGLAVALALSVLPGSRALRPGDCEVCISYLGRFYQDLKDRDVTFSPATIENELIKFCREARGKENRLCYYIGATDDAATKIINEVSKPLAHHIPVEKICEKLKKKDSQICELKYDKQIDLSTVDLKKLRVKELKKILDDWGETCKGCAEKSDYIRKINELMPKYAPKAASARTDL",
# MTP_HUMAN
"MILLAVLFLCFISSYSASVKGHTTGLSLNNDRLYKLTYSTEVLLDRGKGKLQDSVGYRISSNVDVALLWRNPDGDDDQLIQITMKDVNVENVNQQRGEKSIFKGKSPSKIMGKENLEALQRPTLLHLIHGKVKEFYSYQNEAVAIENIKRGLASLFQTQLSSGTTNEVDISGNCKVTYQAHQDKVIKIKALDSCKIARSGFTTPNQVLGVSSKATSVTTYKIEDSFVIAVLAEETHNFGLNFLQTIKGKIVSKQKLELKTTEAGPRLMSGKQAAAIIKAVDSKYTAIPIVGQVFQSHCKGCPSLSELWRSTRKYLQPDNLSKAEAVRNFLAFIQHLRTAKKEEILQILKMENKEVLPQLVDAVTSAQTSDSLEAILDFLDFKSDSSIILQERFLYACGFASHPNEELLRALISKFKGSIGSSDIRETVMIITGTLVRKLCQNEGCKLKAVVEAKKLILGGLEKAEKKEDTRMYLLALKNALLPEGIPSLLKYAEAGEGPISHLATTALQRYDLPFITDEVKKTLNRIYHQNRKVHEKTVRTAAAAIILNNNPSYMDVKNILLSIGELPQEMNKYMLAIVQDILRFEMPASKIVRRVLKEMVAHNYDRFSRSGSSSAYTGYIERSPRSASTYSLDILYSGSGILRRSNLNIFQYIGKAGLHGSQVVIEAQGLEALIAATPDEGEENLDSYAGMSAILFDVQLRPVTFFNGYSDLMSKMLSASGDPISVVKGLILLIDHSQELQLQSGLKANIEVQGGLAIDISGAMEFSLWYRESKTRVKNRVTVVITTDITVDSSFVKAGLETSTETEAGLEFISTVQFSQYPFLVCMQMDKDEAPFRQFEKKYERLSTGRGYVSQKRKESVLAGCEFPLHQENSEMCKVVFAPQPDSTSSGWF",
# CO4A3_HUMAN
"MSARTAPRPQVLLLPLLLVLLAAAPAASKGCVCKDKGQCFCDGAKGEKGEKGFPGPPGSPGQKGFTGPEGLPGPQGPKGFPGLPGLTGSKGVRGISGLPGFSGSPGLPGTPGNTGPYGLVGVPGCSGSKGEQGFPGLPGTLGYPGIPGAAGLKGQKGAPAKEEDIELDAKGDPGLPGAPGPQGLPGPPGFPGPVGPPGPPGFFGFPGAMGPRGPKGHMGERVIGHKGERGVKGLTGPPGPPGTVIVTLTGPDNRTDLKGEKGDKGAMGEPGPPGPSGLPGESYGSEKGAPGDPGLQGKPGKDGVPGFPGSEGVKGNRGFPGLMGEDGIKGQKGDIGPPGFRGPTEYYDTYQEKGDEGTPGPPGPRGARGPQGPSGPPGVPGSPGSSRPGLRGAPGWPGLKGSKGERGRPGKDAMGTPGSPGCAGSPGLPGSPGPPGPPGDIVFRKGPPGDHGLPGYLGSPGIPGVDGPKGEPGLLCTQCPYIPGPPGLPGLPGLHGVKGIPGRQGAAGLKGSPGSPGNTGLPGFPGFPGAQGDPGLKGEKGETLQPEGQVGVPGDPGLRGQPGRKGLDGIPGTPGVKGLPGPKGELALSGEKGDQGPPGDPGSPGSPGPAGPAGPPGYGPQGEPGLQGTQGVPGAPGPPGEAGPRGELSVSTPVPGPPGPPGPPGHPGPQGPPGIPGSLGKCGDPGLPGPDGEPGIPGIGFPGPPGPKGDQGFPGTKGSLGCPGKMGEPGLPGKPGLPGAKGEPAVAMPGGPGTPGFPGERGNSGEHGEIGLPGLPGLPGTPGNEGLDGPRGDPGQPGPPGEQGPPGRCIEGPRGAQGLPGLNGLKGQQGRRGKTGPKGDPGIPGLDRSGFPGETGSPGIPGHQGEMGPLGQRGYPGNPGILGPPGEDGVIGMMGFPGAIGPPGPPGNPGTPGQRGSPGIPGVKGQRGTPGAKGEQGDKGNPGPSEISHVIGDKGEPGLKGFAGNPGEKGNRGVPGMPGLKGLKGLPGPAGPPGPRGDLGSTGNPGEPGLRGIPGSMGNMGMPGSKGKRGTLGFPGRAGRPGLPGIHGLQGDKGEPGYSEGTRPGPPGPTGDPGLPGDMGKKGEMGQPGPPGHLGPAGPEGAPGSPGSPGLPGKPGPHGDLGFKGIKGLLGPPGIRGPPGLPGFPGSPGPMGIRGDQGRDGIPGPAGEKGETGLLRAPPGPRGNPGAQGAKGDRGAPGFPGLPGRKGAMGDAGPRGPTGIEGFPGPPGLPGAIIPGQTGNRGPPGSRGSPGAPGPPGPPGSHVIGIKGDKGSMGHPGPKGPPGTAGDMGPPGRLGAPGTPGLPGPRGDPGFQGFPGVKGEKGNPGFLGSIGPPGPIGPKGPPGVRGDPGTLKIISLPGSPGPPGTPGEPGMQGEPGPPGPPGNLGPCGPRGKPGKDGKPGTPGPAGEKGNKGSKGEPGPAGSDGLPGLKGKRGDSGSPATWTTRGFVFTRHSQTTAIPSCPEGTVPLYSGFSFLFVQGNQRAHGQDLGTLGSCLQRFTTMPFLFCNVNDVCNFASRNDYSYWLSTPALMPMNMAPITGRALEPYISRCTVCEGPAIAIAVHSQTTDIPPCPHGWISLWKGFSFIMFTSAGSEGTGQALASPGSCLEEFRASPFLECHGRGTCNYYSNSYSFWLASLNPERMFRKPIPSTVKAGELEKIISRCQVCMKKRH",
# PDIA2_HUMAN
"MSRQLLPVLLLLLLRASCPWGQEQGARSPSEEPPEEEIPKEDGILVLSRHTLGLALREHPALLVEFYAPWCGHCQALAPEYSKAAAVLAAESMVVTLAKVDGPAQRELAEEFGVTEYPTLKFFRNGNRTHPEEYTGPRDAEGIAEWLRRRVGPSAMRLEDEAAAQALIGGRDLVVIGFFQDLQDEDVATFLALAQDALDMTFGLTDRPRLFQQFGLTKDTVVLFKKFDEGRADFPVDEELGLDLGDLSRFLVTHSMRLVTEFNSQTSAKIFAARILNHLLLFVNQTLAAHRELLAGFGEAAPRFRGQVLFVVVDVAADNEHVLQYFGLKAEAAPTLRLVNLETTKKYAPVDGGPVTAASITAFCHAVLNGQVKPYLLSQEIPPDWDQRPVKTLVGKNFEQVAFDETKNVFVKFYAPWCTHCKEMAPAWEALAEKYQDHEDIIIAELDATANELDAFAVHGFPTLKYFPAGPGRKVIEYKSTRDLETFSKFLDNGGVLPTEEPPEEPAAPFPEPPANSTMGSKEEL",
# DNJC3_HUMAN
"MVAPGSVTSRLGSVFPFLLVLVDLQYEGAECGVNADVEKHLELGKKLLAAGQLADALSQFHAAVDGDPDNYIAYYRRATVFLAMGKSKAALPDLTKVIQLKMDFTAARLQRGHLLLKQGKLDEAEDDFKKVLKSNPSENEEKEAQSQLIKSDEMQRLRSQALNAFGSGDYTAAIAFLDKILEVCVWDAELRELRAECFIKEGEPRKAISDLKAASKLKNDNTEAFYKISTLYYQLGDHELSLSEVRECLKLDQDHKRCFAHYKQVKKLNKLIESAEELIRDGRYTDATSKYESVMKTEPSIAEYTVRSKERICHCFSKDEKPVEAIRVCSEVLQMEPDNVNALKDRAEAYLIEEMYDEAIQDYETAQEHNENDQQIREGLEKAQRLLKQSQKRDYYKILGVKRNAKKQEIIKAYRKLALQWHPDNFQNEEEKKKAEKKFIDIAAAKEVLSDPEMRKKFDDGEDPLDAESQQGGGGNPFHRSWNSWQGFNPFSSGGPFRFKFHFN",
# RCN2_HUMAN
"MRLGPRTAALGLLLLCAAAAGAGKAEELHYPLGERRSDYDREALLGVQEDVDEYVKLGHEEQQKRLQAIIKKIDLDSDGFLTESELSSWIQMSFKHYAMQEAKQQFVEYDKNSDDTVTWDEYNIQMYDRVIDFDENTALDDAEEESFRKLHLKDKKRFEKANQDSGPGLSLEEFIAFEHPEEVDYMTEFVIQEALEEHDKNGDGFVSLEEFLGDYRWDPTANEDPEWILVEKDRFVNDYDKDNDGRLDPQELLPWVVPNNQGIAQEEALHLIDEMDLNGDKKLSEEEILENPDLFLTSEATDYGRQLHDDYFYHDEL",
# RCN1_HUMAN
"MARGGRGRRLGLALGLLLALVLAPRVLRAKPTVRKERVVRPDSELGERPPEDNQSFQYDHEAFLGKEDSKTFDQLTPDESKERLGKIVDRIDNDGDGFVTTEELKTWIKRVQKRYIFDNVAKVWKDYDRDKDDKISWEEYKQATYGYYLGNPAEFHDSSDHHTFKKMLPRDERRFKAADLNGDLTATREEFTAFLHPEEFEHMKEIVVLETLEDIDKNGDGFVDQDEYIADMFSHEENGPEPDWVLSEREQFNEFRDLNKDGKLDKDEIRHWILPQDYDHAQAEARHLVYESDKNKDEKLTKEEILENWNMFVGSQATNYGEDLTKNHDEL",
# P3H1_HUMAN
"MAVRALKLLTTLLAVVAAASQAEVESEAGWGMVTPDLLFAEGTAAYARGDWPGVVLSMERALRSRAALRALRLRCRTQCAADFPWELDPDWSPSPAQASGAAALRDLSFFGGLLRRAACLRRCLGPPAAHSLSEEMELEFRKRSPYNYLQVAYFKINKLEKAVAAAHTFFVGNPEHMEMQQNLDYYQTMSGVKEADFKDLETQPHMQEFRLGVRLYSEEQPQEAVPHLEAALQEYFVAYEECRALCEGPYDYDGYNYLEYNADLFQAITDHYIQVLNCKQNCVTELASHPSREKPFEDFLPSHYNYLQFAYYNIGNYTQAVECAKTYLLFFPNDEVMNQNLAYYAAMLGEEHTRSIGPRESAKEYRQRSLLEKELLFFAYDVFGIPFVDPDSWTPEEVIPKRLQEKQKSERETAVRISQEIGNLMKEIETLVEEKTKESLDVSRLTREGGPLLYEGISLTMNSKLLNGSQRVVMDGVISDHECQELQRLTNVAATSGDGYRGQTSPHTPNEKFYGVTVFKALKLGQEGKVPLQSAHLYYNVTEKVRRIMESYFRLDTPLYFSYSHLVCRTAIEEVQAERKDDSHPVHVDNCILNAETLVCVKEPPAYTFRDYSAILYLNGDFDGGNFYFTELDAKTVTAEVQPQCGRAVGFSSGTENPHGVKAVTRGQRCAIALWFTLDPRHSERDRVQADDLVKMLFSPEEMDLSQEQPLDAQQGPPEPAQESLSGSESKPKDEL",
# P3H2_HUMAN
"MRERIWAPPLLLLLPLLLPPPLWGGPPDSPRRELELEPGPLQPFDLLYASGAAAYYSGDYERAVRDLEAALRSHRRLREIRTRCARHCAARHPLPPPPPGEGPGAELPLFRSLLGRARCYRSCETQRLGGPASRHRVSEDVRSDFQRRVPYNYLQRAYIKLNQLEKAVEAAHTFFVANPEHMEMQQNIENYRATAGVEALQLVDREAKPHMESYNAGVKHYEADDFEMAIRHFEQALREYFVEDTECRTLCEGPQRFEEYEYLGYKAGLYEAIADHYMQVLVCQHECVRELATRPGRLSPIENFLPLHYDYLQFAYYRVGEYVKALECAKAYLLCHPDDEDVLDNVDYYESLLDDSIDPASIEAREDLTMFVKRHKLESELIKSAAEGLGFSYTEPNYWIRYGGRQDENRVPSGVNVEGAEVHGFSMGKKLSPKIDRDLREGGPLLYENITFVYNSEQLNGTQRVLLDNVLSEEQCRELHSVASGIMLVGDGYRGKTSPHTPNEKFEGATVLKALKSGYEGRVPLKSARLFYDISEKARRIVESYFMLNSTLYFSYTHMVCRTALSGQQDRRNDLSHPIHADNCLLDPEANECWKEPPAYTFRDYSALLYMNDDFEGGEFIFTEMDAKTVTASIKPKCGRMISFSSGGENPHGVKAVTKGKRCAVALWFTLDPLYRELERIQADEVIAILDQEQQGKHELNINPKDEL",
# DJC10_HUMAN
"MGVWLNKDDYIRDLKRIILCFLIVYMAILVGTDQDFYSLLGVSKTASSREIRQAFKKLALKLHPDKNPNNPNAHGDFLKINRAYEVLKDEDLRKKYDKYGEKGLEDNQGGQYESWNYYRYDFGIYDDDPEIITLERREFDAAVNSGELWFVNFYSPGCSHCHDLAPTWRDFAKEVDGLLRIGAVNCGDDRMLCRMKGVNSYPSLFIFRSGMAPVKYHGDRSKESLVSFAMQHVRSTVTELWTGNFVNSIQTAFAAGIGWLITFCSKGGDCLTSQTRLRLSGMLDGLVNVGWMDCATQDNLCKSLDITTSTTAYFPPGATLNNKEKNSILFLNSLDAKEIYLEVIHNLPDFELLSANTLEDRLAHHRWLLFFHFGKNENSNDPELKKLKTLLKNDHIQVGRFDCSSAPDICSNLYVFQPSLAVFKGQGTKEYEIHHGKKILYDILAFAKESVNSHVTTLGPQNFPANDKEPWLVDFFAPWCPPCRALLPELRRASNLLYGQLKFGTLDCTVHEGLCNMYNIQAYPTTVVFNQSNIHEYEGHHSAEQILEFIEDLMNPSVVSLTPTTFNELVTQRKHNEVWMVDFYSPWCHPCQVLMPEWKRMARTLTGLINVGSIDCQQYHSFCAQENVQRYPEIRFFPPKSNKAYHYHSYNGWNRDAYSLRIWGLGFLPQVSTDLTPQTFSEKVLQGKNHWVIDFYAPWCGPCQNFAPEFELLARMIKGKVKAGKVDCQAYAQTCQKAGIRAYPTVKFYFYERAKRNFQEEQINTRDAKAIAALISEKLETLRNQGKRNKDEL",
# SUMF2_HUMAN
"MARHGLPLLPLLSLLVGAWLKLGNGQATSMVQLQGGRFLMGTNSPDSRDGDGPVREATVKPFAIDIFPVTNKDFRDFVREKKYRTEAEMFGWSFVFEDFVSDELRNKATQPMKSVLWWLPVEKAFWRQPAGPGSGIRERLEHPVLHVSWNDARAYCAWRGKRLPTEEEWEFAARGGLKGQVYPWGNWFQPNRTNLWQGKFPKGDKAEDGFHGVSPVNAFPAQNNYGLYDLLGNVWEWTASPYQAAEQDMRVLRGASWIDTADGSANHRARVTTRMGNTPDSASDNLGFRCAADAGRPPGEL",
# SUMF1_HUMAN
"MAAPALGLVCGRCPELGLVLLLLLLSLLCGAAGSQEAGTGAGAGSLAGSCGCGTPQRPGAHGSSAAAHRYSREANAPGPVPGERQLAHSKMVPIPAGVFTMGTDDPQIKQDGEAPARRVTIDAFYMDAYEVSNTEFEKFVNSTGYLTEAEKFGDSFVFEGMLSEQVKTNIQQAVAAAPWWLPVKGANWRHPEGPDSTILHRPDHPVLHVSWNDAVAYCTWAGKRLPTEAEWEYSCRGGLHNRLFPWGNKLQPKGQHYANIWQGEFPVTNTGEDGFQGTAPVDAFPPNGYGLYNIVGNAWEWTSDWWTVHHSVEETLNPKGPPSGKDRVKKGGSYMCHRSYCYRYRCAARSQNTPDSSASNLGFRCAADRLPTMD",
# PCSK9_HUMAN
"MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALRSEEDGLAEAPEHGTTATFHRCAKDPWRLPGTYVVVLKEETHLSQSERTARRLQAQAARRGYLTKILHVFHGLLPGFLVKMSGDLLELALKLPHVDYIEEDSSVFAQSIPWNLERITPPRYRADEYQPPDGGSLVEVYLLDTSIQSDHREIEGRVMVTDFENVPEEDGTRFHRQASKCDSHGTHLAGVVSGRDAGVAKGASMRSLRVLNCQGKGTVSGTLIGLEFIRKSQLVQPVGPLVVLLPLAGGYSRVLNAACQRLARAGVVLVTAAGNFRDDACLYSPASAPEVITVGATNAQDQPVTLGTLGTNFGRCVDLFAPGEDIIGASSDCSTCFVSQSGTSQAAAHVAGIAAMMLSAEPELTLAELRQRLIHFSAKDVINEAWFPEDQRVLTPNLVAALPPSTHGAGWQLFCRTVWSAHSGPTRMATAVARCAPDEELLSCSSFSRSGKRRGERMEAQGGKLVCRAHNAFGGEGVYAIARCCLLPQANCSVHTAPPAEASMGTRVHCHQQGHVLTGCSSHWEVEDLGTHKPPVLRPRGQPNQCVGHREASIHASCCHAPGLECKVKEHGIPAPQEQVTVACEEGWTLTGCSALPGTSHVLGAYAVDNTCVVRSRDVSTTGSTSEGAVTAVAICCRSRHLAQASQELQ",
# TXND5_HUMAN
"MPARPGRLLPLLARPAALTALLLLLLGHGGGGRWGARAQEAAAAAADGPPAADGEDGQDPHSKHLYTADMFTHGIQSAAHFVMFFAPWCGHCQRLQPTWNDLGDKYNSMEDAKVYVAKVDCTAHSDVCSAQGVRGYPTLKLFKPGQEAVKYQGPRDFQTLENWMLQTLNEEPVTPEPEVEPPSAPELKQGLYELSASNFELHVAQGDHFIKFFAPWCGHCKALAPTWEQLALGLEHSETVKIGKVDCTQHYELCSGNQVRGYPTLLWFRDGKKVDQYKGKRDLESLREYVESQLQRTETGATETVTPSEAPVLAAEPEADKGTVLALTENNFDDTIAEGITFIKFYAPWCGHCKTLAPTWEELSKKEFPGLAGVKIAEVDCTAERNICSKYSVRGYPTLLLFRGGKKVSEHSGGRDLDSLHRFVLSQAKDEL",
# SELM_HUMAN
"MSLLLPPLALLLLLAALVAPATAATAYRPDWNRLSGLTRARVETCGGUQLNRLKEVKAFVTQDIPFYHNLVMKHLPGADPELVLLGRRYEELERIPLSEMTREEINALVQELGFYRKAAPDAQVPPEYVWAPAKPPEETSDHADL",
# ARSG_HUMAN
"MGWLFLKVLLAGVSFSGFLYPLVDFCISGKTRGQKPNFVIILADDMGWGDLGANWAETKDTANLDKMASEGMRFVDFHAAASTCSPSRASLLTGRLGLRNGVTRNFAVTSVGGLPLNETTLAEVLQQAGYVTGIIGKWHLGHHGSYHPNFRGFDYYFGIPYSHDMGCTDTPGYNHPPCPACPQGDGPSRNLQRDCYTDVALPLYENLNIVEQPVNLSSLAQKYAEKATQFIQRASTSGRPFLLYVALAHMHVPLPVTQLPAAPRGRSLYGAGLWEMDSLVGQIKDKVDHTVKENTFLWFTGDNGPWAQKCELAGSVGPFTGFWQTRQGGSPAKQTTWEGGHRVPALAYWPGRVPVNVTSTALLSVLDIFPTVVALAQASLPQGRRFDGVDVSEVLFGRSQPGHRVLFHPNSGAAGEFGALQTVRLERYKAFYITGGARACDGSTGPELQHKFPLIFNLEDDTAEAVPLERGGAEYQAVLPEVRKVLADVLQDIANDNISSADYTQDPSVTPCCNPYQIACRCQAA",
# GPX7_HUMAN
"MVAATVAAAWLLLWAAACAQQEQDFYDFKAVNIRGKLVSLEKYRGSVSLVVNVASECGFTDQHYRALQQLQRDLGPHHFNVLAFPCNQFGQQEPDSNKEIESFARRTYSVSFPMFSKIAVTGTGAHPAFKYLAQTSGKEPTWNFWKYLVAPDGKVVGAWDPTVSVEEVRPQITALVRKLILLKREDL",
# EDEM2_HUMAN
"MPFRLLIPLGLLCALLPQHHGAPGPDGSAPDPAHYRERVKAMFYHAYDSYLENAFPFDELRPLTCDGHDTWGSFSLTLIDALDTLLILGNVSEFQRVVEVLQDSVDFDIDVNASVFETNIRVVGGLLSAHLLSKKAGVEVEAGWPCSGPLLRMAEEAARKLLPAFQTPTGMPYGTVNLLHGVNPGETPVTCTAGIGTFIVEFATLSSLTGDPVFEDVARVALMRLWESRSDIGLVGNHIDVLTGKWVAQDAGIGAGVDSYFEYLVKGAILLQDKKLMAMFLEYNKAIRNYTRFDDWYLWVQMYKGTVSMPVFQSLEAYWPGLQSLIGDIDNAMRTFLNYYTVWKQFGGLPEFYNIPQGYTVEKREGYPLRPELIESAMYLYRATGDPTLLELGRDAVESIEKISKVECGFATIKDLRDHKLDNRMESFFLAETVKYLYLLFDPTNFIHNNGSTFDAVITPYGECILGAGGYIFNTEAHPIDPAALHCCQRLKEEQWEVEDLMREFYSLKRSRSKFQKNTVSSGPWEPPARPGTLFSPENHDQARERKPAKQKVPLLSCPSQPFTSKLALLGQVFLDSS",
# EDEM3_HUMAN
"MSEAGGRGCGSPVPQRARWRLVAATAAFCLVSATSVWTAGAEPMSREEKQKLGNQVLEMFDHAYGNYMEHAYPADELMPLTCRGRVRGQEPSRGDVDDALGKFSLTLIDSLDTLVVLNKTKEFEDAVRKVLRDVNLDNDVVVSVFETNIRVLGGLLGGHSLAIMLKEKGEYMQWYNDELLQMAKQLGYKLLPAFNTTSGLPYPRINLKFGIRKPEARTGTETDTCTACAGTLILEFAALSRFTGATIFEEYARKALDFLWEKRQRSSNLVGVTINIHTGDWVRKDSGVGAGIDSYYEYLLKAYVLLGDDSFLERFNTHYDAIMRYISQPPLLLDVHIHKPMLNARTWMDALLAFFPGLQVLKGDIRPAIETHEMLYQVIKKHNFLPEAFTTDFRVHWAQHPLRPEFAESTYFLYKATGDPYYLEVGKTLIENLNKYARVPCGFAAMKDVRTGSHEDRMDSFFLAEMFKYLYLLFADKEDIIFDIEDYIFTTEAHLLPLWLSTTNQSISKKNTTSEYTELDDSNFDWTCPNTQILFPNDPLYAQSIREPLKNVVDKSCPRGIIRVEESFRSGAKPPLRARDFMATNPEHLEILKKMGVSLIHLKDGRVQLVQHAIQAASSIDAEDGLRFMQEMIELSSQQQKEQQLPPRAVQIVSHPFFGRVVLTAGPAQFGLDLSKHKETRGFVASSKPSNGCSELTNPEAVMGKIALIQRGQCMFAEKARNIQNAGAIGGIVIDDNEGSSSDTAPLFQMAGDGKDTDDIKIPMLFLFSKEGSIILDAIREYEEVEVLLSDKAKDRDPEMENEEQPSSENDSQNQSGEQISSSSQEVDLVDQESSEENSLNSHPESLSLADMDNAASISPSEQTSNPTENHETTNLNGECTDLDNQLQEQSETEEDSNPNVSWGKKVQPIDSILADWNEDIEAFEMMEKDEL",
# SIL1_HUMAN
"MAPQSLPSSRMAPLGMLLGLLMAACFTFCLSHQNLKEFALTNPEKSSTKETERKETKAEEELDAEVLEVFHPTHEWQALQPGQAVPAGSHVRLNLQTGEREAKLQYEDKFRNNLKGKRLDINTNTYTSQDLKSALAKFKEGAEMESSKEDKARQAEVKRLFRPIEELKKDFDELNVVIETDMQIMVRLINKFNSSSSSLEEKIAALFDLEYYVHQMDNAQDLLSFGGLQVVINGLNSTEPLVKEYAAFVLGAAFSSNPKVQVEAIEGGALQKLLVILATEQPLTAKKKVLFALCSLLRHFPYAQRQFLKLGGLQVLRTLVQEKGTEVLAVRVVTLLYDLVTEKMFAEEEAELTQEMSPEKLQQYRQVHLLPGLWEQGWCEITAHLLALPEHDAREKVLQTLGVLLTTCRDRYRQDPQLGRTLASLQAEYQVLASLELQDGEDEGYFQELLGSVNSLLKELR",
# TOR3A_HUMAN
"MLRGPWRQLWLFFLLLLPGAPEPRGASRPWEGTDEPGSAWAWPGFQRLQEQLRAAGALSKRYWTLFSCQVWPDDCDEDEEAATGPLGWRLPLLGQRYLDLLTTWYCSFKDCCPRGDCRISNNFTGLEWDLNVRLHGQHLVQQLVLRTVRGYLETPQPEKALALSFHGWSGTGKNFVARMLVENLYRDGLMSDCVRMFIATFHFPHPKYVDLYKEQLMSQIRETQQLCHQTLFIFDEAEKLHPGLLEVLGPHLERRAPEGHRAESPWTIFLFLSNLRGDIINEVVLKLLKAGWSREEITMEHLEPHLQAEIVETIDNGFGHSRLVKENLIDYFIPFLPLEYRHVRLCARDAFLSQELLYKEETLDEIAQMMVYVPKEEQLFSSQGCKSISQRINYFLS",
# UGGG2_HUMAN
"MAPAKATNVVRLLLGSTALWLSQLGSGTVAASKSVTAHLAAKWPETPLLLEASEFMAEESNEKFWQFLETVQELAIYKQTESDYSYYNLILKKAGQFLDNLHINLLKFAFSIRAYSPAIQMFQQIAADEPPPDGCNAFVVIHKKHTCKINEIKKLLKKAASRTRPYLFKGDHKFPTNKENLPVVILYAEMGTRTFSAFHKVLSEKAQNEEILYVLRHYIQKPSSRKMYLSGYGVELAIKSTEYKALDDTQVKTVTNTTVEDETETNEVQGFLFGKLKEIYSDLRDNLTAFQKYLIESNKQMMPLKVWELQDLSFQAASQIMSAPVYDSIKLMKDISQNFPIKARSLTRIAVNQHMREEIKENQKDLQVRFKIQPGDARLFINGLRVDMDVYDAFSILDMLKLEGKMMNGLRNLGINGEDMSKFLKLNSHIWEYTYVLDIRHSSIMWINDLENDDLYITWPTSCQKLLKPVFPGSVPSIRRNFHNLVLFIDPAQEYTLDFIKLADVFYSHEVPLRIGFVFILNTDDEVDGANDAGVALWRAFNYIAEEFDISEAFISIVHMYQKVKKDQNILTVDNVKSVLQNTFPHANIWDILGIHSKYDEERKAGASFYKMTGLGPLPQALYNGEPFKHEEMNIKELKMAVLQRMMDASVYLQREVFLGTLNDRTNAIDFLMDRNNVVPRINTLILRTNQQYLNLISTSVTADVEDFSTFFFLDSQDKSAVIAKNMYYLTQDDESIISAVTLWIIADFDKPSGRKLLFNALKHMKTSVHSRLGIIYNPTSKINEENTAISRGILAAFLTQKNMFLRSFLGQLAKEEIATAIYSGDKIKTFLIEGMDKNAFEKKYNTVGVNIFRTHQLFCQDVLKLRPGEMGIVSNGRFLGPLDEDFYAEDFYLLEKITFSNLGEKIKGIVENMGINANNMSDFIMKVDALMSSVPKRASRYDVTFLRENHSVIKTNPQENDMFFNVIAIVDPLTREAQKMAQLLVVLGKIINMKIKLFMNCRGRLSEAPLESFYRFVLEPELMSGANDVSSLGPVAKFLDIPESPLLILNMITPEGWLVETVHSNCDLDNIHLKDTEKTVTAEYELEYLLLEGQCFDKVTEQPPRGLQFTLGTKNKPAVVDTIVMAHHGYFQLKANPGAWILRLHQGKSEDIYQIVGHEGTDSQADLEDIIVVLNSFKSKILKVKVKKETDKIKEDILTDEDEKTKGLWDSIKSFTVSLHKENKKEKDVLNIFSVASGHLYERFLRIMMLSVLRNTKTPVKFWLLKNYLSPTFKEVIPHMAKEYGFRYELVQYRWPRWLRQQTERQRIIWGYKILFLDVLFPLAVDKIIFVDADQIVRHDLKELRDFDLDGAPYGYTPFCDSRREMDGYRFWKTGYWASHLLRRKYHISALYVVDLKKFRRIGAGDRLRSQYQALSQDPNSLSNLDQDLPNNMIYQVAIKSLPQDWLWCETWCDDESKQRAKTIDLCNNPKTKESKLKAAARIVPEWVEYDAEIRQLLDHLENKKQDTILTHDEL",
# UGGG1_HUMAN
"MGCKGDASGACAAGALPVTGVCYKMGVLVVLTVLWLFSSVKADSKAITTSLTTKWFSTPLLLEASEFLAEDSQEKFWNFVEASQNIGSSDHDGTDYSYYHAILEAAFQFLSPLQQNLFKFCLSLRSYSATIQAFQQIAADEPPPEGCNSFFSVHGKKTCESDTLEALLLTASERPKPLLFKGDHRYPSSNPESPVVIFYSEIGSEEFSNFHRQLISKSNAGKINYVFRHYIFNPRKEPVYLSGYGVELAIKSTEYKAKDDTQVKGTEVNTTVIGENDPIDEVQGFLFGKLRDLHPDLEGQLKELRKHLVESTNEMAPLKVWQLQDLSFQTAARILASPVELALVVMKDLSQNFPTKARAITKTAVSSELRTEVEENQKYFKGTLGLQPGDSALFINGLHMDLDTQDIFSLFDVLRNEARVMEGLHRLGIEGLSLHNVLKLNIQPSEADYAVDIRSPAISWVNNLEVDSRYNSWPSSLQELLRPTFPGVIRQIRKNLHNMVFIVDPAHETTAELMNTAEMFLSNHIPLRIGFIFVVNDSEDVDGMQDAGVAVLRAYNYVAQEVDDYHAFQTLTHIYNKVRTGEKVKVEHVVSVLEKKYPYVEVNSILGIDSAYDRNRKEARGYYEQTGVGPLPVVLFNGMPFEREQLDPDELETITMHKILETTTFFQRAVYLGELPHDQDVVEYIMNQPNVVPRINSRILTAERDYLDLTASNNFFVDDYARFTILDSQGKTAAVANSMNYLTKKGMSSKEIYDDSFIRPVTFWIVGDFDSPSGRQLLYDAIKHQKSSNNVRISMINNPAKEISYENTQISRAIWAALQTQTSNAAKNFITKMAKEGAAEALAAGADIAEFSVGGMDFSLFKEVFESSKMDFILSHAVYCRDVLKLKKGQRAVISNGRIIGPLEDSELFNQDDFHLLENIILKTSGQKIKSHIQQLRVEEDVASDLVMKVDALLSAQPKGDPRIEYQFFEDRHSAIKLRPKEGETYFDVVAVVDPVTREAQRLAPLLLVLAQLINMNLRVFMNCQSKLSDMPLKSFYRYVLEPEISFTSDNSFAKGPIAKFLDMPQSPLFTLNLNTPESWMVESVRTPYDLDNIYLEEVDSVVAAEYELEYLLLEGHCYDITTGQPPRGLQFTLGTSANPVIVDTIVMANLGYFQLKANPGAWILRLRKGRSEDIYRIYSHDGTDSPPDADEVVIVLNNFKSKIIKVKVQKKADMVNEDLLSDGTSENESGFWDSFKWGFTGQKTEEVKQDKDDIINIFSVASGHLYERFLRIMMLSVLKNTKTPVKFWFLKNYLSPTFKEFIPYMANEYNFQYELVQYKWPRWLHQQTEKQRIIWGYKILFLDVLFPLVVDKFLFVDADQIVRTDLKELRDFNLDGAPYGYTPFCDSRREMDGYRFWKSGYWASHLAGRKYHISALYVVDLKKFRKIAAGDRLRGQYQGLSQDPNSLSNLDQDLPNNMIHQVPIKSLPQEWLWCETWCDDASKKRAKTIDLCNNPMTKEPKLEAAVRIVPEWQDYDQEIKQLQIRFQKEKETGALYKEKTKEPSREGPQKREEL",
# CATZ_HUMAN
"MARRGPGWRPLLLLVLLAGAAQGGLYFRRGQTCYRPLRGDGLAPLGRSTYPRPHEYLSPADLPKSWDWRNVDGVNYASITRNQHIPQYCGSCWAHASTSAMADRINIKRKGAWPSTLLSVQNVIDCGNAGSCEGGNDLSVWDYAHQHGIPDETCNNYQAKDQECDKFNQCGTCNEFKECHAIRNYTLWRVGDYGSLSGREKMMAEIYANGPISCGIMATERLANYTGGIYAEYQDTTYINHVVSVAGWGISDGTEYWIVRNSWGEPWGERGWLRIVTSTYKDGKGARYNLAIEEHCTFGDPIV",
# DJB11_HUMAN
"MAPQNLSTFCLLLLYLIGAVIAGRDFYKILGVPRSASIKDIKKAYRKLALQLHPDRNPDDPQAQEKFQDLGAAYEVLSDSEKRKQYDTYGEEGLKDGHQSSHGDIFSHFFGDFGFMFGGTPRQQDRNIPRGSDIIVDLEVTLEEVYAGNFVEVVRNKPVARQAPGKRKCNCRQEMRTTQLGPGRFQMTQEVVCDECPNVKLVNEERTLEVEIEPGVRDGMEYPFIGEGEPHVDGEPGDLRFRIKVVKHPIFERRGDDLYTNVTISLVESLVGFEMDITHLDGHKVHISRDKITRPGAKLWKKGEGLPNFDNNNIKGSLIITFDVDFPKEQLTEEAREGIKQLLKQGSVQKVYNGLQGY",
# MINP1_HUMAN
"MLRAPGCLLRTSVAPAAALAAALLSSLARCSLLEPRDPVASSLSPYFGTKTRYEDVNPVLLSGPEAPWRDPELLEGTCTPVQLVALIRHGTRYPTVKQIRKLRQLHGLLQARGSRDGGASSTGSRDLGAALADWPLWYADWMDGQLVEKGRQDMRQLALRLASLFPALFSRENYGRLRLITSSKHRCMDSSAAFLQGLWQHYHPGLPPPDVADMEFGPPTVNDKLMRFFDHCEKFLTEVEKNATALYHVEAFKTGPEMQNILKKVAATLQVPVNDLNADLIQVAFFTCSFDLAIKGVKSPWCDVFDIDDAKVLEYLNDLKQYWKRGYGYTINSRSSCTLFQDIFQHLDKAVEQKQRSQPISSPVILQFGHAETLLPLLSLMGYFKDKEPLTAYNYKKQMHRKFRSGLIVPYASNLIFVLYHCENAKTPKEQFRVQMLLNEKVLPLAYSQETVSFYEDLKNHYKDILQSCQTSEECELARANSTSDEL",
# HYOU1_HUMAN
"MADKVRRQRPRRRVCWALVAVLLADLLALSDTLAVMSVDLGSESMKVAIVKPGVPMEIVLNKESRRKTPVIVTLKENERFFGDSAASMAIKNPKATLRYFQHLLGKQADNPHVALYQARFPEHELTFDPQRQTVHFQISSQLQFSPEEVLGMVLNYSRSLAEDFAEQPIKDAVITVPVFFNQAERRAVLQAARMAGLKVLQLINDNTATALSYGVFRRKDINTTAQNIMFYDMGSGSTVCTIVTYQMVKTKEAGMQPQLQIRGVGFDRTLGGLEMELRLRERLAGLFNEQRKGQRAKDVRENPRAMAKLLREANRLKTVLSANADHMAQIEGLMDDVDFKAKVTRVEFEELCADLFERVPGPVQQALQSAEMSLDEIEQVILVGGATRVPRVQEVLLKAVGKEELGKNINADEAAAMGAVYQAAALSKAFKVKPFVVRDAVVYPILVEFTREVEEEPGIHSLKHNKRVLFSRMGPYPQRKVITFNRYSHDFNFHINYGDLGFLGPEDLRVFGSQNLTTVKLKGVGDSFKKYPDYESKGIKAHFNLDESGVLSLDRVESVFETLVEDSAEEESTLTKLGNTISSLFGGGTTPDAKENGTDTVQEEEESPAEGSKDEPGEQVELKEEAEAPVEDGSQPPPPEPKGDATPEGEKATEKENGDKSEAQKPSEKAEAGPEGVAPAPEGEKKQKPARKRRMVEEIGVELVVLDLPDLPEDKLAQSVQKLQDLTLRDLEKQEREKAANSLEAFIFETQDKLYQPEYQEVSTEEQREEISGKLSAASTWLEDEGVGATTVMLKEKLAELRKLCQGLFFRVEERKKWPERLSALDNLLNHSSMFLKGARLIPEMDQIFTEVEMTTLEKVINETWAWKNATLAEQAKLPATEKPVLLSKDIEAKMMALDREVQYLLNKAKFTKPRPRPKDKNGTRAEPPLNASASDQGEKVIPPAGQTEDAEPISEPEKVETGSEPGDTEPLELGGPGAEPEQKEQSTGQKRPLKNDEL",
# CC88B_HUMAN
"MEGGKGPRLRDFLSGSLATWALGLAGLVGEAEDSEGEEEEEEEEPPLWLEKRFLRLSDGALLLRVLGIIAPSSRGGPRMLRGLDGPAAWRVWNLNHLWGRLRDFYQEELQLLILSPPPDLQTLGFDPLSEEAVEQLEGVLRLLLGASVQCEHRELFIRHIQGLSLEVQSELAAAIQEVTQPGAGVVLALSGPDPGELAPAELEMLSRSLMGTLSKLARERDLGAQRLAELLLEREPLCLRPEAPSRAPAEGPSHHLALQLANAKAQLRRLRQELEEKAELLLDSQAEVQGLEAEIRRLRQEAQALSGQAKRAELYREEAEALRERAGRLPRLQEELRRCRERLQAAEAYKSQLEEERVLSGVLEASKALLEEQLEAARERCARLHETQRENLLLRTRLGEAHAELDSLRHQVDQLAEENVELELELQRSLEPPPGSPGEAPLAGAAPSLQDEVREAEAGRLRTLERENRELRGLLQVLQGQPGGQHPLLEAPREDPVLPVLEEAPQTPVAFDHSPQGLVQKARDGGPQALDLAPPALDSVLEASAECPQAPDSDPQEAESPLQAAAMDPQASDWSPQESGSPVETQESPEKAGRRSSLQSPASVAPPQGPGTKIQAPQLLGGETEGREAPQGELVPEAWGLRQEGPEHKPGPSEPSSVQLEEQEGPNQGLDLATGQAEAREHDQRLEGTVRDPAWQKPQQKSEGALEVQVWEGPIPGESLASGVAEQEALREEVAQLRRKAEALGDELEAQARKLEAQNTEAARLSKELAQARRAEAEAHREAEAQAWEQARLREAVEAAGQELESASQEREALVEALAAAGRERRQWEREGSRLRAQSEAAEERMQVLESEGRQHLEEAERERREKEALQAELEKAVVRGKELGDRLEHLQRELEQAALERQEFLREKESQHQRYQGLEQRLEAELQAAATSKEEALMELKTRALQLEEELFQLRQGPAGLGPKKRAEPQLVETQNVRLIEVERSNAMLVAEKAALQGQLQHLEGQLGSLQGRAQELLLQSQRAQEHSSRLQAEKSVLEIQGQELHRKLEVLEEEVRAARQSQEETRGQQQALLRDHKALAQLQRRQEAELEGLLVRHRDLKANMRALELAHRELQGRHEQLQAQRASVEAQEVALLAERERLMQDGHRQRGLEEELRRLQSEHDRAQMLLAELSRERGELQGERGELRGRLARLELERAQLEMQSQQLRESNQQLDLSACRLTTQCELLTQLRSAQEEENRQLLAEVQALSRENRELLERSLESRDHLHREQREYLDQLNALRREKQKLVEKIMDQYRVLEPVPLPRTKKGSWLADKVKRLMRPRREGGPPGGLRLGADGAGSTESLGGPPETELPEGREADGTGSPSPAPMRRAQSSLCLRDETLAGGQRRKLSSRFPVGRSSESFSPGDTPRQRFRQRHPGPLGAPVSHSKGPGVGWENSAETLQEHETDANREGPEVQEPEKRPLTPSLSQ",
# U17L3_HUMAN
"MGDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSSETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYENASLQCLTYTLPLANYMLSREHSQTCQRPKCCMLCTMQAHITWALHSPGHVIQPSQALASGFHRGKQEDVHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGCWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVKQALEQLVKPEELNGENAYHCGLCLQRAPASNTLTLHTSAKVLILVLKRFSDVAGNKLAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHDGHYFSYVKAQEGQWYKMDDAEVTVCSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRAKQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVGKVEGTLPPNALVIHQSKYKCGMKNHHPEQQSSLLNLSSTTRTDQESMNTGTLASLQGRTRRAKGKNKHSKRALLVCQ",
# U17L4_HUMAN
"MGDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSSETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYENASLQCLTYTLPLANYMLSREHSQTCQRPKCCMLCTMQAHITWALHSPGHVIQPSQALAAGFHRGKQEDVHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGCWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVKQALEQLVKPEELNGENAYHCGLCLQRAPASNTLTLHTSAKVLILVLKRFSDVAGNKLAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHDGYYFSYVKAQEGQWYKMDDAEVTVCSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRPATQGELKRDHPCLQVPELDEHLVERATEESTLDHWKFPQEQNKMKPEFNVRKVEGTLPPNVLVIHQSKYKCGMKNHHPEQQSSLLNLSSMNSTDQESMNTGTLASLQGRTRRSKGKNKHSKRSLLVCQ",
# U17L5_HUMAN
"MEDDSLYLRGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLAKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LF_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIDKNVQYPECLDMKLYMSQTNSGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQWSQWKYRPTRRGAHTHAHTQTHT",
# U17LA_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKPPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYKPPLANYMLFREHSQTCHRHKGCMLCTMQAHITRALHIPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMRKACLPGHKQVDRHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHNSAKVLILVLKRFPDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYSSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGVEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRRVEGTVPPDVLVIHQSKYKCRMKNHHPEQQSSLLNLSSTTPTDQESMNTGTLASLRGRTRRSKGKNKHSKRALLVCQ",
# U17LD_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRLDAAFAEIQRTSLPEKSPLSCETRVDLCDDLVPEARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHPSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTAASITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDRWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# UL17C_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSNRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKMLTLLTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLKLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LB_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQTNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LL_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSNRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKMLTLLTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LI_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQTNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRAKQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LM_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLKLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LH_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTAASITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LJ_HUMAN
"MEEDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQTNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLKLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# U17LK_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# ST2B1_HUMAN
"MDGPAEPQIPGLWDTYEDDISEISQKLPGEYFRYKGVPFPVGLYSLESISLAENTQDVRDDDIFIITYPKSGTTWMIEIICLILKEGDPSWIRSVPIWERAPWCETIVGAFSLPDQYSPRLMSSHLPIQIFTKAFFSSKAKVIYMGRNPRDVVVSLYHYSKIAGQLKDPGTPDQFLRDFLKGEVQFGSWFDHIKGWLRMKGKDNFLFITYEELQQDLQGSVERICGFLGRPLGKEALGSVVAHSTFSAMKANTMSNYTLLPPSLLDHRRGAFLRKGVCGDWKNHFTVAQSEAFDRAYRKQMRGMPTFPWDEDPEEDGSPDPEPSPEPEPKPSLEPNTSLEREPRPNSSPSPSPGQASETPHPRPS",
# PDE2A_HUMAN
"MGQACGHSILCRSQQYPAARPAEPRGQQVFLKPDEPPPPPQPCADSLQDALLSLGSVIDISGLQRAVKEALSAVLPRVETVYTYLLDGESQLVCEDPPHELPQEGKVREAIISQKRLGCNGLGFSDLPGKPLARLVAPLAPDTQVLVMPLADKEAGAVAAVILVHCGQLSDNEEWSLQAVEKHTLVALRRVQVLQQRGPREAPRAVQNPPEGTAEDQKGGAAYTDRDRKILQLCGELYDLDASSLQLKVLQYLQQETRASRCCLLLVSEDNLQLSCKVIGDKVLGEEVSFPLTGCLGQVVEDKKSIQLKDLTSEDVQQLQSMLGCELQAMLCVPVISRATDQVVALACAFNKLEGDLFTDEDEHVIQHCFHYTSTVLTSTLAFQKEQKLKCECQALLQVAKNLFTHLDDVSVLLQEIITEARNLSNAEICSVFLLDQNELVAKVFDGGVVDDESYEIRIPADQGIAGHVATTGQILNIPDAYAHPLFYRGVDDSTGFRTRNILCFPIKNENQEVIGVAELVNKINGPWFSKFDEDLATAFSIYCGISIAHSLLYKKVNEAQYRSHLANEMMMYHMKVSDDEYTKLLHDGIQPVAAIDSNFASFTYTPRSLPEDDTSMAILSMLQDMNFINNYKIDCPTLARFCLMVKKGYRDPPYHNWMHAFSVSHFCYLLYKNLELTNYLEDIEIFALFISCMCHDLDHRGTNNSFQVASKSVLAALYSSEGSVMERHHFAQAIAILNTHGCNIFDHFSRKDYQRMLDLMRDIILATDLAHHLRIFKDLQKMAEVGYDRNNKQHHRLLLCLLMTSCDLSDQTKGWKTTRKIAELIYKEFFSQGDLEKAMGNRPMEMMDREKAYIPELQISFMEHIAMPIYKLLQDLFPKAAELYERVASNREHWTKVSHKFTIRGLPSNNSLDFLDEEYEVPDLDGTRAPINGCCSLDAE",
# P3C2B_HUMAN
"MSSTQGNGEHWKSLESVGISRKELAMAEALQMEYDALSRLRHDKEENRAKQNADPSLISWDEPGVDFYSKPAGRRTDLKLLRGLSGSDPTLNYNSLSPQEGPPNHSTSQGPQPGSDPWPKGSLSGDYLYIFDGSDGGVSSSPGPGDIEGSCKKLSPPPLPPRASIWDTPPLPPRKGSPSSSKISQPSDINTFSLVEQLPGKLLEHRILEEEEVLGGGGQGRLLGSVDYDGINDAITRLNLKSTYDAEMLRDATRGWKEGRGPLDFSKDTSGKPVARSKTMPPQVPPRTYASRYGNRKNATPGKNRRISAAPVGSRPHTVANGHELFEVSEERDEEVAAFCHMLDILRSGSDIQDYFLTGYVWSAVTPSPEHLGDEVNLKVTVLCDRLQEALTFTCNCSSTVDLLIYQTLCYTHDDLRNVDVGDFVLKPCGLEEFLQNKHALGSHEYIQYCRKFDIDIRLQLMEQKVVRSDLARTVNDDQSPSTLNYLVHLQERPVKQTISRQALSLLFDTYHNEVDAFLLADGDFPLKADRVVQSVKAICNALAAVETPEITSALNQLPPCPSRMQPKIQKDPSVLAVRENREKVVEALTAAILDLVELYCNTFNADFQTAVPGSRKHDLVQEACHFARSLAFTVYATHRIPIIWATSYEDFYLSCSLSHGGKELCSPLQTRRAHFSKYLFHLIVWDQQICFPVQVNRLPRETLLCATLYALPIPPPGSSSEANKQRRVPEALGWVTTPLFNFRQVLTCGRKLLGLWPATQENPSARWSAPNFHQPDSVILQIDFPTSAFDIKFTSPPGDKFSPRYEFGSLREEDQRKLKDIMQKESLYWLTDADKKRLWEKRYYCHSEVSSLPLVLASAPSWEWACLPDIYVLLKQWTHMNHQDALGLLHATFPDQEVRRMAVQWIGSLSDAELLDYLPQLVQALKYECYLDSPLVRFLLKRAVSDLRVTHYFFWLLKDGLKDSQFSIRYQYLLAALLCCCGKGLREEFNRQCWLVNALAKLAQQVREAAPSARQGILRTGLEEVKQFFALNGSCRLPLSPSLLVKGIVPRDCSYFNSNAVPLKLSFQNVDPLGENIRVIFKCGDDLRQDMLTLQMIRIMSKIWVQEGLDMRMVIFRCFSTGRGRGMVEMIPNAETLRKIQVEHGVTGSFKDRPLADWLQKHNPGEDEYEKAVENFIYSCAGCCVATYVLGICDRHNDNIMLKTTGHMFHIDFGRFLGHAQMFGNIKRDRAPFVFTSDMAYVINGGDKPSSRFHDFVDLCCQAYNLIRKHTHLFLNLLGLMLSCGIPELSDLEDLKYVYDALRPQDTEANATTYFTRLIESSLGSVATKLNFFIHNLAQMKFTGSDDRLTLSFASRTHTLKSSGRISDVFLCRHEKIFHPNKGYIYVVKVMRENTHEATYIQRTFEEFQELHNKLRLLFPSSHLPSFPSRFVIGRSRGEAVAERRREELNGYIWHLIHAPPEVAECDLVYTFFHPLPRDEKAMGTSPAPKSSDGTWARPVGKVGGEVKLSISYKNNKLFIMVMHIRGLQLLQDGNDPDPYVKIYLLPDPQKTTKRKTKVARKTCNPTYNEMLVYDGIPKGDLQQRELQLSVLSEQGFWENVLLGEVNIRLRELDLAQEKTGWFALGSRSHGTL",
# PDZD2_HUMAN
"MPITQDNAVLHLPLLYQWLQNSLQEGGDGPEQRLCQAAIQKLQEYIQLNFAVDESTVPPDHSPPEMEICTVYLTKELGDTETVGLSFGNIPVFGDYGEKRRGGKKRKTHQGPVLDVGCIWVTELRKNSPAGKSGKVRLRDEILSLNGQLMVGVDVSGASYLAEQCWNGGFIYLIMLRRFKHKAHSTYNGNSSNSSEPGETPTLELGDRTAKKGKRTRKFGVISRPPANKAPEESKGSAGCEVSSDPSTELENGPDPELGNGHVFQLENGPDSLKEVAGPHLERSEVDRGTEHRIPKTDAPLTTSNDKRRFSKGGKTDFQSSDCLAREEVGRIWKMELLKESDGLGIQVSGGRGSKRSPHAIVVTQVKEGGAAHRDGRLSLGDELLVINGHLLVGLSHEEAVAILRSATGMVQLVVASKENSAEDLLRLTSKSLPDLTSSVEDVSSWTDNEDQEADGEEDEGTSSSVQRAMPGTDEPQDVCGAEESKGNLESPKQGSNKIKLKSRLSGGVHRLESVEEYNELMVRNGDPRIRMLEVSRDGRKHSLPQLLDSSSASQEYHIVKKSTRSLSTTQVESPWRLIRPSVISIIGLYKEKGKGLGFSIAGGRDCIRGQMGIFVKTIFPNGSAAEDGRLKEGDEILDVNGIPIKGLTFQEAIHTFKQIRSGLFVLTVRTKLVSPSLTPCSTPTHMSRSASPNFNTSGGASAGGSDEGSSSSLGRKTPGPKDRIVMEVTLNKEPRVGLGIGACCLALENSPPGIYIHSLAPGSVAKMESNLSRGDQILEVNSVNVRHAALSKVHAILSKCPPGPVRLVIGRHPNPKVSEQEMDEVIARSTYQESKEANSSPGLGTPLKSPSLAKKDSLISESELSQYFAHDVPGPLSDFMVAGSEDEDHPGSGCSTSEEGSLPPSTSTHKEPGKPRANSLVTLGSHRASGLFHKQVTVARQASLPGSPQALRNPLLRQRKVGCYDANDASDEEEFDREGDCISLPGALPGPIRPLSEDDPRRVSISSSKGMDVHNQEERPRKTLVSKAISAPLLGSSVDLEESIPEGMVDAASYAANLTDSAEAPKGSPGSWWKKELSGSSSAPKLEYTVRTDTQSPTNTGSPSSPQQKSEGLGSRHRPVARVSPHCKRSEAEAKPSGSQTVNLTGRANDPCDLDSRVQATSVKVTVAGFQPGGAVEKESLGKLTTGDACVSTSCELASALSHLDASHLTENLPKAASELGQQPMTELDSSSDLISSPGKKGAAHPDPSKTSVDTGQVSRPENPSQPASPRVTKCKARSPVRLPHEGSPSPGEKAAAPPDYSKTRSASETSTPHNTRRVAALRGAGPGAEGMTPAGAVLPGDPLTSQEQRQGAPGNHSKALEMTGIHAPESSQEPSLLEGADSVSSRAPQASLSMLPSTDNTKEACGHVSGHCCPGGSRESPVTDIDSFIKELDASAARSPSSQTGDSGSQEGSAQGHPPAGAGGGSSCRAEPVPGGQTSSPRRAWAAGAPAYPQWASQPSVLDSINPDKHFTVNKNFLSNYSRNFSSFHEDSTSLSGLGDSTEPSLSSMYGDAEDSSSDPESLTEAPRASARDGWSPPRSRVSLHKEDPSESEEEQIEICSTRGCPNPPSSPAHLPTQAAICPASAKVLSLKYSTPRESVASPREKAACLPGSYTSGPDSSQPSSLLEMSSQEHETHADISTSQNHRPSCAEETTEVTSASSAMENSPLSKVARHFHSPPIILSSPNMVNGLEHDLLDDETLNQYETSINAAASLSSFSVDVPKNGESVLENLHISESQDLDDLLQKPKMIARRPIMAWFKEINKHNQGTHLRSKTEKEQPLMPARSPDSKIQMVSSSQKKGVTVPHSPPQPKTNLENKDLSKKSPAEMLLTNGQKAKCGPKLKRLSLKGKAKVNSEAPAANAVKAGGTDHRKPLISPQTSHKTLSKAVSQRLHVADHEDPDRNTTAAPRSPQCVLESKPPLATSGPLKPSVSDTSIRTFVSPLTSPKPVPEQGMWSRFHMAVLSEPDRGCPTTPKSPKCRAEGRAPRADSGPVSPAASRNGMSVAGNRQSEPRLASHVAADTAQPRPTGEKGGNIMASDRLERTNQLKIVEISAEAVSETVCGNKPAESDRRGGCLAQGNCQEKSEIRLYRQVAESSTSHPSSLPSHASQAEQEMSRSFSMAKLASSSSSLQTAIRKAEYSQGKSSLMSDSRGVPRNSIPGGPSGEDHLYFTPRPATRTYSMPAQFSSHFGREGHPPHSLGRSRDSQVPVTSSVVPEAKASRGGLPSLANGQGIYSVKPLLDTSRNLPATDEGDIISVQETSCLVTDKIKVTRRHYCYEQNWPHESTSFFSVKQRIKSFENLANADRPVAKSGASPFLSVSSKPPIGRRSSGSIVSGSLGHPGDAAARLLRRSLSSCSENQSEAGTLLPQMAKSPSIMTLTISRQNPPETSSKGSDSELKKSLGPLGIPTPTMTLASPVKRNKSSVRHTQPSPVSRSKLQELRALSMPDLDKLCSEDYSAGPSAVLFKTELEITPRRSPGPPAGGVSCPEKGGNRACPGGSGPKTSAAETPSSASDTGEAAQDLPFRRSWSVNLDQLLVSAGDQQRLQSVLSSVGSKSTILTLIQEAKAQSENEEDVCFIVLNRKEGSGLGFSVAGGTDVEPKSITVHRVFSQGAASQEGTMNRGDFLLSVNGASLAGLAHGNVLKVLHQAQLHKDALVVIKKGMDQPRPSARQEPPTANGKGLLSRKTIPLEPGIGRSVAVHDALCVEVLKTSAGLGLSLDGGKSSVTGDGPLVIKRVYKGGAAEQAGIIEAGDEILAINGKPLVGLMHFDAWNIMKSVPEGPVQLLIRKHRNSS",
# YKT6_HUMAN
"MKLYSLSVLYKGEAKVVLLKAAYDVSSFSFFQRSSVQEFMTFTSQLIVERSSKGTRASVKEQDYLCHVYVRNDSLAGVVIADNEYPSRVAFTLLEKVLDEFSKQVDRIDWPVGSPATIHYPALDGHLSRYQNPREADPMTKVQAELDETKIILHNTMESLLERGEKLDDLVSKSEVLGTQSKAFYKTARKQNSCCAIM",
# AT8B1_HUMAN
"MSTERDSETTFDEDSQPNDEVVPYSDDETEDELDDQGSAVEPEQNRVNREAEENREPFRKECTWQVKANDRKYHEQPHFMNTKFLCIKESKYANNAIKTYKYNAFTFIPMNLFEQFKRAANLYFLALLILQAVPQISTLAWYTTLVPLLVVLGVTAIKDLVDDVARHKMDKEINNRTCEVIKDGRFKVAKWKEIQVGDVIRLKKNDFVPADILLLSSSEPNSLCYVETAELDGETNLKFKMSLEITDQYLQREDTLATFDGFIECEEPNNRLDKFTGTLFWRNTSFPLDADKILLRGCVIRNTDFCHGLVIFAGADTKIMKNSGKTRFKRTKIDYLMNYMVYTIFVVLILLSAGLAIGHAYWEAQVGNSSWYLYDGEDDTPSYRGFLIFWGYIIVLNTMVPISLYVSVEVIRLGQSHFINWDLQMYYAEKDTPAKARTTTLNEQLGQIHYIFSDKTGTLTQNIMTFKKCCINGQIYGDHRDASQHNHNKIEQVDFSWNTYADGKLAFYDHYLIEQIQSGKEPEVRQFFFLLAVCHTVMVDRTDGQLNYQAASPDEGALVNAARNFGFAFLARTQNTITISELGTERTYNVLAILDFNSDRKRMSIIVRTPEGNIKLYCKGADTVIYERLHRMNPTKQETQDALDIFANETLRTLCLCYKEIEEKEFTEWNKKFMAASVASTNRDEALDKVYEEIEKDLILLGATAIEDKLQDGVPETISKLAKADIKIWVLTGDKKETAENIGFACELLTEDTTICYGEDINSLLHARMENQRNRGGVYAKFAPPVQESFFPPGGNRALIITGSWLNEILLEKKTKRNKILKLKFPRTEEERRMRTQSKRRLEAKKEQRQKNFVDLACECSAVICCRVTPKQKAMVVDLVKRYKKAITLAIGDGANDVNMIKTAHIGVGISGQEGMQAVMSSDYSFAQFRYLQRLLLVHGRWSYIRMCKFLRYFFYKNFAFTLVHFWYSFFNGYSAQTAYEDWFITLYNVLYTSLPVLLMGLLDQDVSDKLSLRFPGLYIVGQRDLLFNYKRFFVSLLHGVLTSMILFFIPLGAYLQTVGQDGEAPSDYQSFAVTIASALVITVNFQIGLDTSYWTFVNAFSIFGSIALYFGIMFDFHSAGIHVLFPSAFQFTGTASNALRQPYIWLTIILAVAVCLLPVVAIRFLSMTIWPSESDKIQKHRKRLKAEEQWQRRQQVFRRGVSTRRSAYAFSHQRGYADLISSGRSIRKKRSPLDAIVADGTAEYRRTGDS",
# TPPC3_HUMAN
"MSRQANRGTESKKMSSELFTLTYGALVTQLCKDYENDEDVNKQLDKMGFNIGVRLIEDFLARSNVGRCHDFRETADVIAKVAFKMYLGITPSITNWSPAGDEFSLILENNPLVDFVELPDNHSSLIYSNLLCGVLRGALEMVQMAVEAKFVQDTLKGDGVTEIRMRFIRRIEDNLPAGEE",
# NCK2_HUMAN
"MTEEVIVIAKWDYTAQQDQELDIKKNERLWLLDDSKTWWRVRNAANRTGYVPSNYVERKNSLKKGSLVKNLKDTLGLGKTRRKTSARDASPTPSTDAEYPANGSGADRIYDLNIPAFVKFAYVAEREDELSLVKGSRVTVMEKCSDGWWRGSYNGQIGWFPSNYVLEEVDEAAAESPSFLSLRKGASLSNGQGSRVLHVVQTLYPFSSVTEEELNFEKGETMEVIEKPENDPEWWKCKNARGQVGLVPKNYVVVLSDGPALHPAHAPQISYTGPSSSGRFAGREWYYGNVTRHQAECALNERGVEGDFLIRDSESSPSDFSVSLKASGKNKHFKVQLVDNVYCIGQRRFHTMDELVEHYKKAPIFTSEHGEKLYLVRALQ",
# PLIN1_HUMAN
"MAVNKGLTLLDGDLPEQENVLQRVLQLPVVSGTCECFQKTYTSTKEAHPLVASVCNAYEKGVQSASSLAAWSMEPVVRRLSTQFTAANELACRGLDHLEEKIPALQYPPEKIASELKDTISTRLRSARNSISVPIASTSDKVLGAALAGCELAWGVARDTAEFAANTRAGRLASGGADLALGSIEKVVEYLLPPDKEESAPAPGHQQAQKSPKAKPSLLSRVGALTNTLSRYTVQTMARALEQGHTVAMWIPGVVPLSSLAQWGASVAMQAVSRRRSEVRVPWLHSLAAAQEEDHEDQTDTEGEDTEEEEELETEENKFSEVAALPGPRGLLGGVAHTLQKTLQTTISAVTWAPAAVLGMAGRVLHLTPAPAVSSTKGRAMSLSDALKGVTDNVVDTVVHYVPLPRLSLMEPESEFRDIDNPPAEVERREAERRASGAPSAGPEPAPRLAQPRRSLRSAQSPGAPPGPGLEDEVATPAAPRPGFPAVPREKPKRRVSDSFFRPSVMEPILGRTHYSQLRKKS",
# PRKN_HUMAN
"MIVFVRFNSSHGFPVEVDSDTSIFQLKEVVAKRQGVPADQLRVIFAGKELRNDWTVQNCDLDQQSIVHIVQRPWRKGQEMNATGGDDPRNAAGGCEREPQSLTRVDLSSSVLPGDSVGLAVILHTDSRKDSPPAGSPAGRSIYNSFYVYCKGPCQRVQPGKLRVQCSTCRQATLTLTQGPSCWDDVLIPNRMSGECQSPHCPGTSAEFFFKCGAHPTSDKETSVALHLIATNSRNITCITCTDVRSPVLVFQCNSRHVICLDCFHLYCVTRLNDRQFVHDPQLGYSLPCVAGCPNSLIKELHHFRILGEEQYNRYQQYGAEECVLQMGGVLCPRPGCGAGLLPEPDQRKVTCEGGNGLGCGFAFCRECKEAYHEGECSAVFEASGTTTQAYRVDERAAEQARWEAASKETIKKTTKPCPRCHVPVEKNGGCMHMKCPQPQCRLEWCWNCGCEWNRVCMGDHWFDV",
# CUBN_HUMAN
"MMNMSLPFLWSLLTLLIFAEVNGEAGELELQRQKRSINLQQPRMATERGNLVFLTGSAQNIEFRTGSLGKIKLNDEDLSECLHQIQKNKEDIIELKGSAIGLPQNISSQIYQLNSKLVDLERKFQGLQQTVDKKVCSSNPCQNGGTCLNLHDSFFCICPPQWKGPLCSADVNECEIYSGTPLSCQNGGTCVNTMGSYSCHCPPETYGPQCASKYDDCEGGSVARCVHGICEDLMREQAGEPKYSCVCDAGWMFSPNSPACTLDRDECSFQPGPCSTLVQCFNTQGSFYCGACPTGWQGNGYICEDINECEINNGGCSVAPPVECVNTPGSSHCQACPPGYQGDGRVCTLTDICSVSNGGCHPDASCSSTLGSLPLCTCLPGYTGNGYGPNGCVQLSNICLSHPCLNGQCIDTVSGYFCKCDSGWTGVNCTENINECLSNPCLNGGTCVDGVDSFSCECTRLWTGALCQVPQQVCGESLSGINGSFSYRSPDVGYVHDVNCFWVIKTEMGKVLRITFTFFRLESMDNCPHEFLQVYDGDSSSAFQLGRFCGSSLPHELLSSDNALYFHLYSEHLRNGRGFTVRWETQQPECGGILTGPYGSIKSPGYPGNYPPGRDCVWIVVTSPDLLVTFTFGTLSLEHHDDCNKDYLEIRDGPLYQDPLLGKFCTTFSVPPLQTTGPFARIHFHSDSQISDQGFHITYLTSPSDLRCGGNYTDPEGELFLPELSGPFTHTRQCVYMMKQPQGEQIQINFTHVELQCQSDSSQNYIEVRDGETLLGKVCGNGTISHIKSITNSVWIRFKIDASVEKASFRAVYQVACGDELTGEGVIRSPFFPNVYPGERTCRWTIHQPQSQVILLNFTVFEIGSSAHCETDYVEIGSSSILGSPENKKYCGTDIPSFITSVYNFLYVTFVKSSSTENHGFMAKFSAEDLACGEILTESTGTIQSPGHPNVYPHGINCTWHILVQPNHLIHLMFETFHLEFHYNCTNDYLEVYDTDSETSLGRYCGKSIPPSLTSSGNSLMLVFVTDSDLAYEGFLINYEAISAATACLQDYTDDLGTFTSPNFPNNYPNNWECIYRITVRTGQLIAVHFTNFSLEEAIGNYYTDFLEIRDGGYEKSPLLGIFYGSNLPPTIISHSNKLWLKFKSDQIDTRSGFSAYWDGSSTGCGGNLTTSSGTFISPNYPMPYYHSSECYWWLKSSHGSAFELEFKDFHLEHHPNCTLDYLAVYDGPSSNSHLLTQLCGDEKPPLIRSSGDSMFIKLRTDEGQQGRGFKAEYRQTCENVVIVNQTYGILESIGYPNPYSENQHCNWTIRATTGNTVNYTFLAFDLEHHINCSTDYLELYDGPRQMGRYCGVDLPPPGSTTSSKLQVLLLTDGVGRREKGFQMQWFVYGCGGELSGATGSFSSPGFPNRYPPNKECIWYIRTDPGSSIQLTIHDFDVEYHSRCNFDVLEIYGGPDFHSPRIAQLCTQRSPENPMQVSSTGNELAIRFKTDLSINGRGFNASWQAVTGGCGGIFQAPSGEIHSPNYPSPYRSNTDCSWVIRVDRNHRVLLNFTDFDLEPQDSCIMAYDGLSSTMSRLARTCGREQLANPIVSSGNSLFLRFQSGPSRQNRGFRAQFRQACGGHILTSSFDTVSSPRFPANYPNNQNCSWIIQAQPPLNHITLSFTHFELERSTTCARDFVEILDGGHEDAPLRGRYCGTDMPHPITSFSSALTLRFVSDSSISAGGFHTTVTASVSACGGTFYMAEGIFNSPGYPDIYPPNVECVWNIVSSPGNRLQLSFISFQLEDSQDCSRDFVEIREGNATGHLVGRYCGNSFPLNYSSIVGHTLWVRFISDGSGSGTGFQATFMKIFGNDNIVGTHGKVASPFWPENYPHNSNYQWTVNVNASHVVHGRILEMDIEEIQNCYYDKLRIYDGPSIHARLIGAYCGTQTESFSSTGNSLTFHFYSDSSISGKGFLLEWFAVDAPDGVLPTIAPGACGGFLRTGDAPVFLFSPGWPDSYSNRVDCTWLIQAPDSTVELNILSLDIESHRTCAYDSLVIRDGDNNLAQQLAVLCGREIPGPIRSTGEYMFIRFTSDSSVTRAGFNASFHKSCGGYLHADRGIITSPKYPETYPSNLNCSWHVLVQSGLTIAVHFEQPFQIPNGDSSCNQGDYLVLRNGPDICSPPLGPPGGNGHFCGSHASSTLFTSDNQMFVQFISDHSNEGQGFKIKYEAKSLACGGNVYIHDADSAGYVTSPNHPHNYPPHADCIWILAAPPETRIQLQFEDRFDIEVTPNCTSNYLELRDGVDSDAPILSKFCGTSLPSSQWSSGEVMYLRFRSDNSPTHVGFKAKYSIAQCGGRVPGQSGVVESIGHPTLPYRDNLFCEWHLQGLSGHYLTISFEDFNLQNSSGCEKDFVEIWDNHTSGNILGRYCGNTIPDSIDTSSNTAVVRFVTDGSVTASGFRLRFESSMEECGGDLQGSIGTFTSPNYPNPNPHGRICEWRITAPEGRRITLMFNNLRLATHPSCNNEHVIVFNGIRSNSPQLEKLCSSVNVSNEIKSSGNTMKVIFFTDGSRPYGGFTASYTSSEDAVCGGSLPNTPEGNFTSPGYDGVRNYSRNLNCEWTLSNPNQGNSSISIHFEDFYLESHQDCQFDVLEFRVGDADGPLMWRLCGPSKPTLPLVIPYSQVWIHFVTNERVEHIGFHAKYSFTDCGGIQIGDSGVITSPNYPNAYDSLTHCSSLLEAPQGHTITLTFSDFDIEPHTTCAWDSVTVRNGGSPESPIIGQYCGNSNPRTIQSGSNQLVVTFNSDHSLQGGGFYATWNTQTLGCGGIFHSDNGTIRSPHWPQNFPENSRCSWTAITHKSKHLEISFDNNFLIPSGDGQCQNSFVKVWAGTEEVDKALLATGCGNVAPGPVITPSNTFTAVFQSQEAPAQGFSASFVSRCGSNFTGPSGYIISPNYPKQYDNNMNCTYVIEANPLSVVLLTFVSFHLEARSAVTGSCVNDGVHIIRGYSVMSTPFATVCGDEMPAPLTIAGPVLLNFYSNEQITDFGFKFSYRIISCGGVFNFSSGIITSPAYSYADYPNDMHCLYTITVSDDKVIELKFSDFDVVPSTSCSHDYLAIYDGANTSDPLLGKFCGSKRPPNVKSSNNSMLLVFKTDSFQTAKGWKMSFRQTLGPQQGCGGYLTGSNNTFASPDSDSNGMYDKNLNCVWIIIAPVNKVIHLTFNTFALEAASTRQRCLYDYVKLYDGDSENANLAGTFCGSTVPAPFISSGNFLTVQFISDLTLEREGFNATYTIMDMPCGGTYNATWTPQNISSPNSSDPDVPFSICTWVIDSPPHQQVKITVWALQLTSQDCTQNYLQLQDSPQGHGNSRFQFCGRNASAVPVFYSSMSTAMVIFKSGVVNRNSRMSFTYQIADCNRDYHKAFGNLRSPGWPDNYDNDKDCTVTLTAPQNHTISLFFHSLGIENSVECRNDFLEVRNGSNSNSPLLGKYCGTLLPNPVFSQNNELYLRFKSDSVTSDRGYEIIWTSSPSGCGGTLYGDRGSFTSPGYPGTYPNNTYCEWVLVAPAGRLVTINFYFISIDDPGDCVQNYLTLYDGPNASSPSSGPYCGGDTSIAPFVASSNQVFIKFHADYARRPSAFRLTWDS",
# HNRPQ_HUMAN
"MATEHVNGNGTEEPMDTTSAVIHSENFQTLLDAGLPQKVAEKLDEIYVAGLVAHSDLDERAIEALKEFNEDGALAVLQQFKDSDLSHVQNKSAFLCGVMKTYRQREKQGTKVADSSKGPDEAKIKALLERTGYTLDVTTGQRKYGGPPPDSVYSGQQPSVGTEIFVGKIPRDLFEDELVPLFEKAGPIWDLRLMMDPLTGLNRGYAFVTFCTKEAAQEAVKLYNNHEIRSGKHIGVCISVANNRLFVGSIPKSKTKEQILEEFSKVTEGLTDVILYHQPDDKKKNRGFCFLEYEDHKTAAQARRRLMSGKVKVWGNVGTVEWADPIEDPDPEVMAKVKVLFVRNLANTVTEEILEKAFSQFGKLERVKKLKDYAFIHFDERDGAVKAMEEMNGKDLEGENIEIVFAKPPDQKRKERKAQRQAAKNQMYDDYYYYGPPHMPPPTRGRGRGGRGGYGYPPDYYGYEDYYDYYGYDYHNYRGGYEDPYYGYEDFQVGARGRGGRGARGAAPSRGRGAAPPRGRAGYSQRGGPGSARGVRGARGGAQQQRGRGVRGARGGRGGNVGGKRKADGYNQPDSKRRQTNNQNWGSQPIAQQPLQGGDHSGNYGYKSENQEFYQDTFGQQWK",
# USO1_HUMAN
"MNFLRGVMGGQSAGPQHTEAETIQKLCDRVASSTLLDDRRNAVRALKSLSKKYRLEVGIQAMEHLIHVLQTDRSDSEIIGYALDTLYNIISNEEEEEVEENSTRQSEDLGSQFTEIFIKQQENVTLLLSLLEEFDFHVRWPGVKLLTSLLKQLGPQVQQIILVSPMGVSRLMDLLADSREVIRNDGVLLLQALTRSNGAIQKIVAFENAFERLLDIISEEGNSDGGIVVEDCLILLQNLLKNNNSNQNFFKEGSYIQRMKPWFEVGDENSGWSAQKVTNLHLMLQLVRVLVSPTNPPGATSSCQKAMFQCGLLQQLCTILMATGVPADILTETINTVSEVIRGCQVNQDYFASVNAPSNPPRPAIVVLLMSMVNERQPFVLRCAVLYCFQCFLYKNQKGQGEIVSTLLPSTIDATGNSVSAGQLLCGGLFSTDSLSNWCAAVALAHALQENATQKEQLLRVQLATSIGNPPVSLLQQCTNILSQGSKIQTRVGLLMLLCTWLSNCPIAVTHFLHNSANVPFLTGQIAENLGEEEQLVQGLCALLLGISIYFNDNSLESYMKEKLKQLIEKRIGKENFIEKLGFISKHELYSRASQKPQPNFPSPEYMIFDHEFTKLVKELEGVITKAIYKSSEEDKKEEEVKKTLEQHDNIVTHYKNMIREQDLQLEELRQQVSTLKCQNEQLQTAVTQQVSQIQQHKDQYNLLKIQLGKDNQHQGSYSEGAQMNGIQPEEIGRLREEIEELKRNQELLQSQLTEKDSMIENMKSSQTSGTNEQSSAIVSARDSEQVAELKQELATLKSQLNSQSVEITKLQTEKQELLQKTEAFAKSVEVQGETETIIATKTTDVEGRLSALLQETKELKNEIKALSEERTAIKEQLDSSNSTIAILQTEKDKLELEITDSKKEQDDLLVLLADQDQKILSLKNKLKDLGHPVEEEDELESGDQEDEDDESEDPGKDLDHI",
# DIAP2_HUMAN
"MEQPGAAASGAGGGSEEPGGGRSNKRSAGNRAANEEETKNKPKLNIQIKTLADDVRDRITSFRKSTVKKEKPLIQHPIDSQVAMSEFPAAQPLYDERSLNLSEKEVLDLFEKMMEDMNLNEEKKAPLRNKDFTTKREMVVQYISATAKSGGLKNSKHECTLSSQEYVHELRSGISDEKLLNCLESLRVSLTSNPVSWVNNFGHEGLGLLLDELEKLLDKKQQENIDKKNQYKLIQCLKAFMNNKFGLQRILGDERSLLLLARAIDPKQPNMMTEIVKILSAICIVGEENILDKLLGAITTAAERNNRERFSPIVEGLENQEALQLQVACMQFINALVTSPYELDFRIHLRNEFLRSGLKTMLPDLKEKENDELDIQLKVFDENKEDDLTELSHRLNDIRAEMDDMNEVYHLLYNMLKDTAAENYFLSILQHFLLIRNDYYIRPQYYKIIEECVSQIVLHCSGMDPDFKYRQRLDIDLTHLIDSCVNKAKVEESEQKAAEFSKKFDEEFTARQEAQAELQKRDEKIKELEAEIQQLRTQAQVLSSSSGIPGPPAAPPLPGVGPPPPPPAPPLPGGAPLPPPPPPLPGMMGIPPPPPPPLLFGGPPPPPPLGGVPPPPGISLNLPYGMKQKKMYKPEVSMKRINWSKIEPTELSENCFWLRVKEDKFENPDLFAKLALNFATQIKVQKNAEALEEKKTGPTKKKVKELRILDPKTAQNLSIFLGSYRMPYEDIRNVILEVNEDMLSEALIQNLVKHLPEQKILNELAELKNEYDDLCEPEQFGVVMSSVKMLQPRLSSILFKLTFEEHINNIKPSIIAVTLACEELKKSESFNRLLELVLLVGNYMNSGSRNAQSLGFKINFLCKIRDTKSADQKTTLLHFIADICEEKYRDILKFPEELEHVESASKVSAQILKSNLASMEQQIVHLERDIKKFPQAENQHDKFVEKMTSFTKTAREQYEKLSTMHNNMMKLYENLGEYFIFDSKTVSIEEFFGDLNNFRTLFLEAVRENNKRREMEEKTRRAKLAKEKAEQEKLERQKKKKQLIDINKEGDETGVMDNLLEALQSGAAFRDRRKRIPRNPDNRRVPLERSRSRHNGAISSK",
# CLN5_HUMAN
"MAQEVDTAQGAEMRRGAGAARGRASWCWALALLWLAVVPGWSRVSGIPSRRHWPVPYKRFDFRPKPDPYCQAKYTFCPTGSPIPVMEGDDDIEVFRLQAPVWEFKYGDLLGHLKIMHDAIGFRSTLTGKNYTMEWYELFQLGNCTFPHLRPEMDAPFWCNQGAACFFEGIDDVHWKENGTLVQVATISGNMFNQMAKWVKQDNETGIYYETWNVKASPEKGAETWFDSYDCSKFVLRTFNKLAEFGAEFKNIETNYTRIFLYSGEPTYLGNETSVFGPTGNKTLGLAIKRFYYPFKPHLPTKEFLLSLLQIFDAVIVHKQFYLFYNFEYWFLPMKFPFIKITYEEIPLPIRNKTLSGL",
# TPC6A_HUMAN
"MADTVLFEFLHTEMVAELWAHDPDPGPGGQKMSLSVLEGMGFRVGQALGERLPRETLAFREELDVLKFLCKDLWVAVFQKQMDSLRTNHQGTYVLQDNSFPLLLPMASGLQYLEEAPKFLAFTCGLLRGALYTLGIESVVTASVAALPVCKFQVVIPKS",
# PDE9A_HUMAN
"MGSGSSSYRPKAIYLDIDGRIQKVIFSKYCNSSDIMDLFCIATGLPRNTTISLLTTDDAMVSIDPTMPANSERTPYKVRPVAIKQLSAGVEDKRTTSRGQSAERPLRDRRVVGLEQPRREGAFESGQVEPRPREPQGCYQEGQRIPPEREELIQSVLAQVAEQFSRAFKINELKAEVANHLAVLEKRVELEGLKVVEIEKCKSDIKKMREELAARSSRTNCPCKYSFLDNHKKLTPRRDVPTYPKYLLSPETIEALRKPTFDVWLWEPNEMLSCLEHMYHDLGLVRDFSINPVTLRRWLFCVHDNYRNNPFHNFRHCFCVAQMMYSMVWLCSLQEKFSQTDILILMTAAICHDLDHPGYNNTYQINARTELAVRYNDISPLENHHCAVAFQILAEPECNIFSNIPPDGFKQIRQGMITLILATDMARHAEIMDSFKEKMENFDYSNEEHMTLLKMILIKCCDISNEVRPMEVAEPWVDCLLEEYFMQSDREKSEGLPVAPFMDRDKVTKATAQIGFIKFVLIPMFETVTKLFPMVEEIMLQPLWESRDRYEELKRIDDAMKELQKKTDSLTSGATEKSRERSRDVKNSEGDCA",
# SRP72_HUMAN
"MASGGSGGVSVPALWSEVNRYGQNGDFTRALKTVNKILQINKDDVTALHCKVVCLIQNGSFKEALNVINTHTKVLANNSLSFEKAYCEYRLNRIENALKTIESANQQTDKLKELYGQVLYRLERYDECLAVYRDLVRNSQDDYDEERKTNLSAVVAAQSNWEKVVPENLGLQEGTHELCYNTACALIGQGQLNQAMKILQKAEDLCRRSLSEDTDGTEEDPQAELAIIHGQMAYILQLQGRTEEALQLYNQIIKLKPTDVGLLAVIANNIITINKDQNVFDSKKKVKLTNAEGVEFKLSKKQLQAIEFNKALLAMYTNQAEQCRKISASLQSQSPEHLLPVLIQAAQLCREKQHTKAIELLQEFSDQHPENAAEIKLTMAQLKISQGNISKACLILRSIEELKHKPGMVSALVTMYSHEEDIDSAIEVFTQAIQWYQNHQPKSPAHLSLIREAANFKLKYGRKKEAISDLQQLWKQNPKDIHTLAQLISAYSLVDPEKAKALSKHLPSSDSMSLKVDVEALENSAGATYIRKKGGKVTGDSQPKEQGQGDLKKKKKKKKGKLPKNYDPKVTPDPERWLPMRERSYYRGRKKGKKKDQIGKGTQGATAGASSELDASKTVSSPPTSPRPGSAATVSASTSNIIPPRHQKPAGAPATKKKQQQKKKKGGKGGW",
# BAIP3_HUMAN
"MRPRGAAFAAGPPGDLHLGTAIGFAGAIWRSRSPAMSTLLDIKSSVLRQVQVCPSFRRRTEQDPGSASADPQEPATGAWKPGDGVEFFAHMRLMLKKGEGRQGLPCLEVPLRSGSPAPPEPVDPSLGLRALAPEEVEMLYEEALYTVLYRAGTMGPDQVDDEEALLSYLQQVFGTSLEEHTEAIERVRKAKAPTYALKVSVMRAKNLLAKDPNGFSDPYCMLGILPASDATREPRAQKEQRFGFRKGSKRGGPLPAKCIQVTEVKSSTLNPVWKEHFLFEIEDVSTDQLHLDIWDHDDDVSLVEACRKLNEVIGLKGMGRYFKQIVKSARANGTAGPTEDHTDDFLGCLNIPVREVPVAGVDRWFKLEPRSSASRVQGHCHLVLKLITTQRDTAMSQRGRSGFLSHLLLLSHLLRLEHSAEEPNSSSWRGELSTPAATILCLHGAQSNLSPLQLAVLHWQVSSRHHQTCTLDYSYLLGLLEDMQAHWEEAPSLPQEQEESLADSLSAFSEFGLQLLRQLRDYFPATNSTAVHRLELLLKCLGKLQLFQPSFEICPFESELNMDIAAALKRGNREWYDRILNDKSPREQPGPQRLPGLVVLADAVYDDLQFCYSVYASLFHSILNVDVFTLTFRQLERLVAEEAWVLTEELSPKMTLEVASGLFELYLTLADLQRFWDSIPGRDSRSLALAGIHAPFLPAVKLWFQVLRDQAKWRLQGAVDMDTLEPVDASSRHSSSAATAGLCLSHIQELWVRLAWPDPAQAQGLGTQLGQDVCEATLFYTELLRKKVDTQPGAAGEAVSEALCVVLNNVELVRKAAGQALKGLAWPEGATGPEGVLPRPLLSCTQALDDDLQREAHTVTAHLTSKMVGDIRKYVQHISLSPDSIQNDEAVAPLMKYLDEKLALLNASLVKGNLSRVLEALWELLLQAILQALGANRDVSADFYSRFHFTLEALVSFFHAEGQGLPLESLRDGSYKRLKEELRLHKCSTRECIEQFYLDKLKQRTLEQNRFGRLSVRCHYEAAEQRLAVEVLHAADLLPLDANGLSDPFVIVELGPPHLFPLVRSQRTQVKTRTLHPVYDELFYFSVPAEACRRRAACVLFTVMDHDWLSTNDFAGEAALGLGGVTGVARPQVGGGARAGQPVTLHLCRPRAQVRSALRRLEGRTSKEAQEFVKKLKELEKCMEADP",
# AHSA1_HUMAN
"MAKWGEGDPRWIVEERADATNVNNWHWTERDASNWSTDKLKTLFLAVQVQNEEGKCEVTEVSKLDGEASINNRKGKLIFFYEWSVKLNWTGTSKSGVQYKGHVEIPNLSDENSVDEVEISVSLAKDEPDTNLVALMKEEGVKLLREAMGIYISTLKTEFTQGMILPTMNGESVDPVGQPALKTEERKAKPAPSKTQARPVGVKIPTCKITLKETFLTSPEELYRVFTTQELVQAFTHAPATLEADRGGKFHMVDGNVSGEFTDLVPEKHIVMKWRFKSWPEGHFATITLTFIDKNGETELCMEGRGIPAPEEERTRQGWQRYYFEGIKQTFGYGARLF",
# PSMG1_HUMAN
"MAATFFGEVVKAPCRAGTEDEEEEEEGRRETPEDREVRLQLARKREVRLLRRQTKTSLEVSLLEKYPCSKFIIAIGNNAVAFLSSFVMNSGVWEEVGCAKLWNEWCRTTDTTHLSSTEAFCVFYHLKSNPSVFLCQCSCYVAEDQQYQWLEKVFGSCPRKNMQITILTCRHVTDYKTSESTGSLPSPFLRALKTQNFKDSACCPLLEQPNIVHDLPAAVLSYCQVWKIPAILYLCYTDVMKLDLITVEAFKPILSTRSLKGLVKNIPQSTEILKKLMTTNEIQSNIYT",
# STAU1_HUMAN
"MSQVQVQVQNPSAALSGSQILNKNQSLLSQPLMSIPSTTSSLPSENAGRPIQNSALPSASITSTSAAAESITPTVELNALCMKLGKKPMYKPVDPYSRMQSTYNYNMRGGAYPPRYFYPFPVPPLLYQVELSVGGQQFNGKGKTRQAAKHDAAAKALRILQNEPLPERLEVNGRESEEENLNKSEISQVFEIALKRNLPVNFEVARESGPPHMKNFVTKVSVGEFVGEGEGKSKKISKKNAAIAVLEELKKLPPLPAVERVKPRIKKKTKPIVKPQTSPEYGQGINPISRLAQIQQAKKEKEPEYTLLTERGLPRRREFVMQVKVGNHTAEGTGTNKKVAKRNAAENMLEILGFKVPQAQPTKPALKSEEKTPIKKPGDGRKVTFFEPGSGDENGTSNKEDEFRMPYLSHQQLPAGILPMVPEVAQAVGVSQGHHTKDFTRAAPNPAKATVTAMIARELLYGGTSPTAETILKNNISSGHVPHGPLTRPSEQLDYLSRVQGFQVEYKDFPKNNKNEFVSLINCSSQPPLISHGIGKDVESCHDMAALNILKLLSELDQQSTEMPRTGNGPMSVCGRC",
# G6B_HUMAN
"MAVFLQLLPLLLSRAQGNPGASLDGRPGDRVNLSCGGVSHPIRWVWAPSFPACKGLSKGRRPILWASSSGTPTVPPLQPFVGRLRSLDSGIRRLELLLSAGDSGTFFCKGRHEDESRTVLHVLGDRTYCKAPGPTHGSVYPQLLIPLLGAGLVLGLGALGLVWWLHRRLPPQPIRPLPRFAPLVKTEPQRPVKEEEPKIPGDLDQEPSLLYADLDHLALSRPRRLSTADPADASTIYAVVV",
# OAS1_HUMAN
"MMDLRNTPAKSLDKFIEDYLLPDTCFRMQINHAIDIICGFLKERCFRGSSYPVCVSKVVKGGSSGKGTTLRGRSDADLVVFLSPLTTFQDQLNRRGEFIQEIRRQLEACQRERAFSVKFEVQAPRWGNPRALSFVLSSLQLGEGVEFDVLPAFDALGQLTGGYKPNPQIYVKLIEECTDLQKEGEFSTCFTELQRDFLKQRPTKLKSLIRLVKHWYQNCKKKLGKLPPQYALELLTVYAWERGSMKTHFNTAQGFRTVLELVINYQQLCIYWTKYYDFKNPIIEKYLRRQLTKPRPVILDPADPTGNLGGGDPKGWRQLAQEAEAWLNYPCFKNWDGSPVSSWILLAESNSADDETDDPRRYQKYGYIGTHEYPHFSHRPSTLQAASTPQAEEDWTCTIL",
# FOS_HUMAN
"MMFSGFNADYEASSSRCSSASPAGDSLSYYHSPADSFSSMGSPVNAQDFCTDLAVSSANFIPTVTAISTSPDLQWLVQPALVSSVAPSQTRAPHPFGVPAPSAGAYSRAGVVKTMTGGRAQSIGRRGKVEQLSPEEEEKRRIRRERNKMAAAKCRNRRRELTDTLQAETDQLEDEKSALQTEIANLLKEKEKLEFILAAHRPACKIPDDLGFPEEMSVASLDLTGGLPEVATPESEEAFTLPLLNDPEPKPSVEPVKSISSMELKTEPFDDFLFPASSRPSGSETARSVPDMDLSGSFYAADWEPLHSGSLGMGPMATELEPLCTPVVTCTPSCTAYTSSFVFTYPEADSFPSCAAAHRKGSSSNEPSSDSLSSPTLLAL",
# CATA_HUMAN
"MADSRDPASDQMQHWKEQRAAQKADVLTTGAGNPVGDKLNVITVGPRGPLLVQDVVFTDEMAHFDRERIPERVVHAKGAGAFGYFEVTHDITKYSKAKVFEHIGKKTPIAVRFSTVAGESGSADTVRDPRGFAVKFYTEDGNWDLVGNNTPIFFIRDPILFPSFIHSQKRNPQTHLKDPDMVWDFWSLRPESLHQVSFLFSDRGIPDGHRHMNGYGSHTFKLVNANGEAVYCKFHYKTDQGIKNLSVEDAARLSQEDPDYGIRDLFNAIATGKYPSWTFYIQVMTFNQAETFPFNPFDLTKVWPHKDYPLIPVGKLVLNRNPVNYFAEVEQIAFDPSNMPPGIEASPDKMLQGRLFAYPDTHRHRLGPNYLHIPVNCPYRARVANYQRDGPMCMQDNQGGAPNYYPNSFGAPEQQPSALEHSIQYSGEVRRFNTANDDNVTQVRAFYVNVLNEEQRKRLCENIAGHLKDAQIFIQKKAVKNFTEVHPDYGSHIQALLDKYNAEKPKNAIHTFVQSGSHLAAREKANL",
# PRIO_HUMAN
"MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGGNRYPPQGGGGWGQPHGGGWGQPHGGGWGQPHGGGWGQPHGGGWGQGGGTHSQWNKPSKPKTNMKHMAGAAAAGAVVGGLGGYMLGSAMSRPIIHFGSDYEDRYYRENMHRYPNQVYYRPMDEYSNQNNFVHDCVNITIKQHTVTTTTKGENFTETDVKMMERVVEQMCITQYERESQAYYQRGSSMVLFSSPPVILLISFLIFLIVG",
# P53_HUMAN
"MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD",
# APOD_HUMAN
"MVMLLLLLSALAGLFGAAEGQAFHLGKCPNPPVQENFDVNKYLGRWYEIEKIPTTFENGRCIQANYSLMENGKIKVLNQELRADGTVNQIEGEATPVNLTEPAKLEVKFSWFMPSAPYWILATDYENYALVYSCTCIIQLFHVDFAWILARNPNLPPETVDSLKNILTSNNIDVKKMTVTDQVNCPKLS",
# RLA0_HUMAN
"MPREDRATWKSNYFLKIIQLLDDYPKCFIVGADNVGSKQMQQIRMSLRGKAVVLMGKNTMMRKAIRGHLENNPALEKLLPHIRGNVGFVFTKEDLTEIRDMLLANKVPAAARAGAIAPCEVTVPAQNTGLGPEKTSFFQALGITTKISRGTIEILSDVQLIKTGDKVGASEATLLNMLNISPFSFGLVIQQVFDNGSIYNPEVLDITEETLHSRFLEGVRNVASVCLQIGYPTVASVPHSIINGYKRVLALSVETDYTFPLAEKVKAFLADPSAFVAAAPVAAATTAAPAAAAAPAKVEAKEESEESDEDMGFGLFD",
# IFIT2_HUMAN
"MSENNKNSLESSLRQLKCHFTWNLMEGENSLDDFEDKVFYRTEFQNREFKATMCNLLAYLKHLKGQNEAALECLRKAEELIQQEHADQAEIRSLVTWGNYAWVYYHMGRLSDVQIYVDKVKHVCEKFSSPYRIESPELDCEEGWTRLKCGGNQNERAKVCFEKALEKKPKNPEFTSGLAIASYRLDNWPPSQNAIDPLRQAIRLNPDNQYLKVLLALKLHKMREEGEEEGEGEKLVEEALEKAPGVTDVLRSAAKFYRRKDEPDKAIELLKKALEYIPNNAYLHCQIGCCYRAKVFQVMNLRENGMYGKRKLLELIGHAVAHLKKADEANDNLFRVCSILASLHALADQYEDAEYYFQKEFSKELTPVAKQLLHLRYGNFQLYQMKCEDKAIHHFIEGVKINQKSREKEKMKDKLQKIAKMRLSKNGADSEALHVLAFLQELNEKMQQADEDSERGLESGSLIPSASSWNGE",
# U17L7_HUMAN
"MEDDSLYLGGDWQFNHFSKLTSSRLDAAFAEIQRTSLSEKSPLSSETRFDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQKIGNTFYVNVSLQCLTYTLPLSNYMLSREDSQTCHLHKCCMFCTMQAHITWALHSPGHVIQPSQVLAAGFHRGEQEDAHEFLMFTVDAMKKACLPGHKQLDHHSKDTTLIHQIFGAYWRSQIKYLHCHGVSDTFDPYLDIALDIQAAQSVKQALEQLVKPKELNGENAYHCGLCLQKAPASKTLTLPTSAKVLILVLKRFSDVTGNKLAKNVQYPKCRDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASGITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRPATQGELKRDHPCLQVPELDEHLVERATQESTLDHWKFPQEQNKTKPEFNVRKVEGTLPPNVLVIHQSKYKCGMKNHHPEQQSSLLNLSSTKPTDQESMNTGTLASLQGSTRRSKGNNKHSKRSLLVCQ",
# U17L8_HUMAN
"MEDDSLYLGGEWQFNHFSKLTSPRPDAAFAEIQRTSLPEKSPLSSETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYLNASLQCLTYTPPLANYMLSREHSQTCQRPKCCMLCTMQAHITWALHSPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGCWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVKQALEQLVKPEELNGENAYPCGLCLQRAPASNTLTLHTSAKVLILVLKRFCDVTGNKLAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHNGYYFSYVKAQEGQWYKMDDAEVTACSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRPATQGELKRDHPCLQVPELDEHLVERATEESTLDHWKFPQEQNKMKPEFNVRKVEGTLPPNVLVIHQSKYKCGMKNHHPEQQSSLLNLSSMNSTDQESMNTGTLASLQGRTRRSKGKNKHSKRSLLVCQ",
# TPC2A_HUMAN
"MSGSFYFVIVGHHDNPVFEMEFLPAGKAESKDDHRHLNQFIAHAALDLVDENMWLSNNMYLKTVDKFNEWFVSAFVTAGHMRFIMLHDIRQEDGIKNFFTDVYDLYIKFSMNPFYEPNSPIRSSAFDRKVQFLGKKHLLS",
# HS71A_HUMAN
"MAKAAAIGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVALNPQNTVFDAKRLIGRKFGDPVVQSDMKHWPFQVINDGDKPKVQVSYKGETKAFYPEEISSMVLTKMKEIAEAYLGYPVTNAVITVPAYFNDSQRQATKDAGVIAGLNVLRIINEPTAAAIAYGLDRTGKGERNVLIFDLGGGTFDVSILTIDDGIFEVKATAGDTHLGGEDFDNRLVNHFVEEFKRKHKKDISQNKRAVRRLRTACERAKRTLSSSTQASLEIDSLFEGIDFYTSITRARFEELCSDLFRSTLEPVEKALRDAKLDKAQIHDLVLVGGSTRIPKVQKLLQDFFNGRDLNKSINPDEAVAYGAAVQAAILMGDKSENVQDLLLLDVAPLSLGLETAGGVMTALIKRNSTIPTKQTQIFTTYSDNQPGVLIQVYEGERAMTKDNNLLGRFELSGIPPAPRGVPQIEVTFDIDANGILNVTATDKSTGKANKITITNDKGRLSKEEIERMVQEAEKYKAEDEVQRERVSAKNALESYAFNMKSAVEDEGLKGKISEADKKKVLDKCQEVISWLDANTLAEKDEFEHKRKELEQVCNPIISGLYQGAGGPGPGGFGAQGPKGGSGSGPTIEEVD",
# HS71B_HUMAN
"MAKAAAIGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTDTERLIGDAAKNQVALNPQNTVFDAKRLIGRKFGDPVVQSDMKHWPFQVINDGDKPKVQVSYKGETKAFYPEEISSMVLTKMKEIAEAYLGYPVTNAVITVPAYFNDSQRQATKDAGVIAGLNVLRIINEPTAAAIAYGLDRTGKGERNVLIFDLGGGTFDVSILTIDDGIFEVKATAGDTHLGGEDFDNRLVNHFVEEFKRKHKKDISQNKRAVRRLRTACERAKRTLSSSTQASLEIDSLFEGIDFYTSITRARFEELCSDLFRSTLEPVEKALRDAKLDKAQIHDLVLVGGSTRIPKVQKLLQDFFNGRDLNKSINPDEAVAYGAAVQAAILMGDKSENVQDLLLLDVAPLSLGLETAGGVMTALIKRNSTIPTKQTQIFTTYSDNQPGVLIQVYEGERAMTKDNNLLGRFELSGIPPAPRGVPQIEVTFDIDANGILNVTATDKSTGKANKITITNDKGRLSKEEIERMVQEAEKYKAEDEVQRERVSAKNALESYAFNMKSAVEDEGLKGKISEADKKKVLDKCQEVISWLDANTLAEKDEFEHKRKELEQVCNPIISGLYQGAGGPGPGGFGAQGPKGGSGSGPTIEEVD",
# NCK1_HUMAN
"MAEEVVVVAKFDYVAQQEQELDIKKNERLWLLDDSKSWWRVRNSMNKTGFVPSNYVERKNSARKASIVKNLKDTLGIGKVKRKPSVPDSASPADDSFVDPGERLYDLNMPAYVKFNYMAEREDELSLIKGTKVIVMEKCSDGWWRGSYNGQVGWFPSNYVTEEGDSPLGDHVGSLSEKLAAVVNNLNTGQVLHVVQALYPFSSSNDEELNFEKGDVMDVIEKPENDPEWWKCRKINGMVGLVPKNYVTVMQNNPLTSGLEPSPPQCDYIRPSLTGKFAGNPWYYGKVTRHQAEMALNERGHEGDFLIRDSESSPNDFSVSLKAQGKNKHFKVQLKETVYCIGQRKFSTMEELVEHYKKAPIFTSEQGEKLYLVKHLS",
# KPCA_HUMAN
"MADVFPGNDSTASQDVANRFARKGALRQKNVHEVKDHKFIARFFKQPTFCSHCTDFIWGFGKQGFQCQVCCFVVHKRCHEFVTFSCPGADKGPDTDDPRSKHKFKIHTYGSPTFCDHCGSLLYGLIHQGMKCDTCDMNVHKQCVINVPSLCGMDHTEKRGRIYLKAEVADEKLHVTVRDAKNLIPMDPNGLSDPYVKLKLIPDPKNESKQKTKTIRSTLNPQWNESFTFKLKPSDKDRRLSVEIWDWDRTTRNDFMGSLSFGVSELMKMPASGWYKLLNQEEGEYYNVPIPEGDEEGNMELRQKFEKAKLGPAGNKVISPSEDRKQPSNNLDRVKLTDFNFLMVLGKGSFGKVMLADRKGTEELYAIKILKKDVVIQDDDVECTMVEKRVLALLDKPPFLTQLHSCFQTVDRLYFVMEYVNGGDLMYHIQQVGKFKEPQAVFYAAEISIGLFFLHKRGIIYRDLKLDNVMLDSEGHIKIADFGMCKEHMMDGVTTRTFCGTPDYIAPEIIAYQPYGKSVDWWAYGVLLYEMLAGQPPFDGEDEDELFQSIMEHNVSYPKSLSKEAVSVCKGLMTKHPAKRLGCGPEGERDVREHAFFRRIDWEKLENREIQPPFKPKVCGKGAENFDKFFTRGQPVLTPPDQLVIANIDQSDFEGFSYVNPQFVHPILQSAV",
# CAN2_HUMAN
"MAGIAAKLAKDREAAEGLGSHDRAIKYLNQDYEALRNECLEAGTLFQDPSFPAIPSALGFKELGPYSSKTRGIEWKRPTEICADPQFIIGGATRTDICQGALGDCWLLAAIASLTLNEEILARVVPLNQSFQENYAGIFHFQFWQYGEWVEVVVDDRLPTKDGELLFVHSAEGSEFWSALLEKAYAKINGCYEALSGGATTEGFEDFTGGIAEWYELKKPPPNLFKIIQKALQKGSLLGCSIDITSAADSEAITFQKLVKGHAYSVTGAEEVESNGSLQKLIRIRNPWGEVEWTGRWNDNCPSWNTIDPEERERLTRRHEDGEFWMSFSDFLRHYSRLEICNLTPDTLTSDTYKKWKLTKMDGNWRRGSTAGGCRNYPNTFWMNPQYLIKLEEEDEDEEDGESGCTFLVGLIQKHRRRQRKMGEDMHTIGFGIYEVPEELSGQTNIHLSKNFFLTNRARERSDTFINLREVLNRFKLPPGEYILVPSTFEPNKDGDFCIRVFSEKKADYQAVDDEIEANLEEFDISEDDIDDGFRRLFAQLAGEDAEISAFELQTILRRVLAKRQDIKSDGFSIETCKIMVDMLDSDGSGKLGLKEFYILWTKIQKYQKIYREIDVDRSGTMNSYEMRKALEEAGFKMPCQLHQVIVARFADDQLIIDFDNFVRCLVRLETLFKIFKQLDPENTGTIELDLISWLCFSVL",
# PTN2_HUMAN
"MPTTIEREFEELDTQRRWQPLYLEIRNESHDYPHRVAKFPENRNRNRYRDVSPYDHSRVKLQNAENDYINASLVDIEEAQRSYILTQGPLPNTCCHFWLMVWQQKTKAVVMLNRIVEKESVKCAQYWPTDDQEMLFKETGFSVKLLSEDVKSYYTVHLLQLENINSGETRTISHFHYTTWPDFGVPESPASFLNFLFKVRESGSLNPDHGPAVIHCSAGIGRSGTFSLVDTCLVLMEKGDDINIKQVLLNMRKYRMGLIQTPDQLRFSYMAIIEGAKCIKGDSSIQKRWKELSKEDLSPAFDHSPNKIMTEKYNGNRIGLEEEKLTGDRCTGLSSKMQDTMEENSESALRKRIREDRKATTAQKVQQMKQRLNENERKRKRWLYWQPILTKMGFMSVILVGAFVGWTLFFQQNAL",
# ICAL_HUMAN
"MNPTETKAIPVSQQMEGPHLPNKKKHKKQAVKTEPEKKSQSTKLSVVHEKKSQEGKPKEHTEPKSLPKQASDTGSNDAHNKKAVSRSAEQQPSEKSTEPKTKPQDMISAGGESVAGITAISGKPGDKKKEKKSLTPAVPVESKPDKPSGKSGMDAALDDLIDTLGGPEETEEENTTYTGPEVSDPMSSTYIEELGKREVTIPPKYRELLAKKEGITGPPADSSKPIGPDDAIDALSSDFTCGSPTAAGKKTEKEESTEVLKAQSAGTVRSAAPPQEKKRKVEKDTMSDQALEALSASLGTRQAEPELDLRSIKEVDEAKAKEEKLEKCGEDDETIPSEYRLKPATDKDGKPLLPEPEEKPKPRSESELIDELSEDFDRSECKEKPSKPTEKTEESKAAAPAPVSEAVCRTSMCSIQSAPPEPATLKGTVPDDAVEALADSLGKKEADPEDGKPVMDKVKEKAKEEDREKLGEKEETIPPDYRLEEVKDKDGKPLLPKESKEQLPPMSEDFLLDALSEDFSGPQNASSLKFEDAKLAAAISEVVSQTPASTTQAGAPPRDTSQSDKDLDDALDKLSDSLGQRQPDPDENKPMEDKVKEKAKAEHRDKLGERDDTIPPEYRHLLDDNGQDKPVKPPTKKSEDSKKPADDQDPIDALSGDLDSCPSTTETSQNTAKDKCKKAASSSKAPKNGGKAKDSAKTTEETSKPKDD",
# ACOHC_HUMAN
"MSNPFAHLAEPLDPVQPGKKFFNLNKLEDSRYGRLPFSIRVLLEAAIRNCDEFLVKKQDIENILHWNVTQHKNIEVPFKPARVILQDFTGVPAVVDFAAMRDAVKKLGGDPEKINPVCPADLVIDHSIQVDFNRRADSLQKNQDLEFERNRERFEFLKWGSQAFHNMRIIPPGSGIIHQVNLEYLARVVFDQDGYYYPDSLVGTDSHTTMIDGLGILGWGVGGIEAEAVMLGQPISMVLPQVIGYRLMGKPHPLVTSTDIVLTITKHLRQVGVVGKFVEFFGPGVAQLSIADRATIANMCPEYGATAAFFPVDEVSITYLVQTGRDEEKLKYIKKYLQAVGMFRDFNDPSQDPDFTQVVELDLKTVVPCCSGPKRPQDKVAVSDMKKDFESCLGAKQGFKGFQVAPEHHNDHKTFIYDNTEFTLAHGSVVIAAITSCTNTSNPSVMLGAGLLAKKAVDAGLNVMPYIKTSLSPGSGVVTYYLQESGVMPYLSQLGFDVVGYGCMTCIGNSGPLPEPVVEAITQGDLVAVGVLSGNRNFEGRVHPNTRANYLASPPLVIAYAIAGTIRIDFEKEPLGVNAKGQQVFLKDIWPTRDEIQAVERQYVIPGMFKEVYQKIETVNESWNALATPSDKLFFWNSKSTYIKSPPFFENLTLDLQPPKSIVDAYVLLNLGDSVTTDHISPAGNIARNSPAARYLTNRGLTPREFNSYGSRRGNDAVMARGTFANIRLLNRFLNKQAPQTIHLPSGEILDVFDAAERYQQAGLPLIVLAGKEYGAGSSRDWAAKGPFLLGIKAVLAESYERIHRSNLVGMGVIPLEYLPGENADALGLTGQERYTIIIPENLKPQMKVQVKLDTGKTFQAVMRFDTDVELTYFLNGGILNYMIRKMAK",
# TGM2_HUMAN
"MAEELVLERCDLELETNGRDHHTADLCREKLVVRRGQPFWLTLHFEGRNYEASVDSLTFSVVTGPAPSQEAGTKARFPLRDAVEEGDWTATVVDQQDCTLSLQLTTPANAPIGLYRLSLEASTGYQGSSFVLGHFILLFNAWCPADAVYLDSEEERQEYVLTQQGFIYQGSAKFIKNIPWNFGQFEDGILDICLILLDVNPKFLKNAGRDCSRRSSPVYVGRVVSGMVNCNDDQGVLLGRWDNNYGDGVSPMSWIGSVDILRRWKNHGCQRVKYGQCWVFAAVACTVLRCLGIPTRVVTNYNSAHDQNSNLLIEYFRNEFGEIQGDKSEMIWNFHCWVESWMTRPDLQPGYEGWQALDPTPQEKSEGTYCCGPVPVRAIKEGDLSTKYDAPFVFAEVNADVVDWIQQDDGSVHKSINRSLIVGLKISTKSVGRDEREDITHTYKYPEGSSEEREAFTRANHLNKLAEKEETGMAMRIRVGQSMNMGSDFDVFAHITNNTAEEYVCRLLLCARTVSYNGILGPECGTKYLLNLNLEPFSEKSVPLCILYEKYRDCLTESNLIKVRALLVEPVINSYLLAERDLYLENPEIKIRILGEPKQKRKLVAEVSLQNPLPVALEGCTFTVEGAGLTEEQKTVEIPDPVEAGEEVKVRMDLLPLHMGLHKLVVNFESDKLKAVKGFRNVIIGPA",
# RS3_HUMAN
"MAVQISKKRKFVADGIFKAELNEFLTRELAEDGYSGVEVRVTPTRTEIIILATRTQNVLGEKGRRIRELTAVVQKRFGFPEGSVELYAEKVATRGLCAIAQAESLRYKLLGGLAVRRACYGVLRFIMESGAKGCEVVVSGKLRGQRAKSMKFVDGLMIHSGDPVNYYVDTAVRHVLLRQGVLGIKVKIMLPWDPTGKIGPKKPLPDHVSIVEPKDEILPTTPISEQKGGKPEPPAMPQPVPTA",
# EF1B_HUMAN
"MGFGDLKSPAGLQVLNDYLADKSYIEGYVPSQADVAVFEAVSSPPPADLCHALRWYNHIKSYEKEKASLPGVKKALGKYGPADVEDTTGSGATDSKDDDDIDLFGSDDEEESEEAKRLREERLAQYESKKAKKPALVAKSSILLDVKPWDDETDMAKLEECVRSIQADGLVWGSSKLVPVGYGIKKLQIQCVVEDDKVGTDMLEEQITAFEDYVQSMDVAAFNKI",
# EF1G_HUMAN
"MAAGTLYTYPENWRAFKALIAAQYSGAQVRVLSAPPHFHFGQTNRTPEFLRKFPAGKVPAFEGDDGFCVFESNAIAYYVSNEELRGSTPEAAAQVVQWVSFADSDIVPPASTWVFPTLGIMHHNKQATENAKEEVRRILGLLDAYLKTRTFLVGERVTLADITVVCTLLWLYKQVLEPSFRQAFPNTNRWFLTCINQPQFRAVLGEVKLCEKMAQFDAKKFAETQPKKDTPRKEKGSREEKQKPQAERKEEKKAAAPAPEEEMDECEQALAAEPKAKDPFAHLPKSTFVLDEFKRKYSNEDTLSVALPYFWEHFDKDGWSLWYSEYRFPEELTQTFMSCNLITGMFQRLDKLRKNAFASVILFGTNNSSSISGVWVFRGQELAFPLSPDWQVDYESYTWRKLDPGSEETQTLVREYFSWEGAFQHVGKAFNQGKIFK",
# RL10_HUMAN
"MGRRPARCYRYCKNKPYPKSRFCRGVPDAKIRIFDLGRKKAKVDEFPLCGHMVSDEYEQLSSEALEAARICANKYMVKSCGKDGFHIRVRLHPFHVIRINKMLSCAGADRLQTGMRGAFGKPQGTVARVHIGQVIMSIRTKLQNKEHVIEALRRAKFKFPGRQKIHISKKWGFTKFNADEFEDMVAEKRLIPDGCGVKYIPSRGPLDKWRALHS",
# RABP2_HUMAN
"MPNFSGNWKIIRSENFEELLKVLGVNVMLRKIAVAAASKPAVEIKQEGDTFYIKTSTTVRTTEINFKVGEEFEEQTVDGRPCKSLVKWESENKMVCEQKLLKGEGPKTSWTRELTNDGELILTMTADDVVCTRVYVRE",
# EF1D_HUMAN
"MATNFLAHEKIWFDKFKYDDAERRFYEQMNGPVAGASRQENGASVILRDIARARENIQKSLAGSSGPGASSGTSGDHGELVVRIASLEVENQSLRGVVQELQQAISKLEARLNVLEKSSPGHRATAPQTQHVSPMRQVEPPAKKPATPAEDDEDDDIDLFGSDNEEEDKEAAQLREERLRQYAEKKAKKPALVAKSSILLDVKPWDDETDMAQLEACVRSIQLDGLVWGASKLVPVGYGIRKLQIQCVVEDDKVGTDLLEEEITKFEEHVQSVDIAAFNKI",
# OAS2_HUMAN
"MGNGESQLSSVPAQKLGWFIQEYLKPYEECQTLIDEMVNTICDVLQEPEQFPLVQGVAIGGSYGRKTVLRGNSDGTLVLFFSDLKQFQDQKRSQRDILDKTGDKLKFCLFTKWLKNNFEIQKSLDGFTIQVFTKNQRISFEVLAAFNALSLNDNPSPWIYRELKRSLDKTNASPGEFAVCFTELQQKFFDNRPGKLKDLILLIKHWHQQCQKKIKDLPSLSPYALELLTVYAWEQGCRKDNFDIAEGVRTVLELIKCQEKLCIYWMVNYNFEDETIRNILLHQLQSARPVILDPVDPTNNVSGDKICWQWLKKEAQTWLTSPNLDNELPAPSWNVLPAPLFTTPGHLLDKFIKEFLQPNKCFLEQIDSAVNIIRTFLKENCFRQSTAKIQIVRGGSTAKGTALKTGSDADLVVFHNSLKSYTSQKNERHKIVKEIHEQLKAFWREKEEELEVSFEPPKWKAPRVLSFSLKSKVLNESVSFDVLPAFNALGQLSSGSTPSPEVYAGLIDLYKSSDLPGGEFSTCFTVLQRNFIRSRPTKLKDLIRLVKHWYKECERKLKPKGSLPPKYALELLTIYAWEQGSGVPDFDTAEGFRTVLELVTQYQQLCIFWKVNYNFEDETVRKFLLSQLQKTRPVILDPAEPTGDVGGGDRWCWHLLAKEAKEWLSSPCFKDGTGNPIPPWKVPTMQTPGSCGARIHPIVNEMFSSRSHRILNNNSKRNF",
# AL3A1_HUMAN
"MSKISEAVKRARAAFSSGRTRPLQFRIQQLEALQRLIQEQEQELVGALAADLHKNEWNAYYEEVVYVLEEIEYMIQKLPEWAADEPVEKTPQTQQDELYIHSEPLGVVLVIGTWNYPFNLTIQPMVGAIAAGNSVVLKPSELSENMASLLATIIPQYLDKDLYPVINGGVPETTELLKERFDHILYTGSTGVGKIIMTAAAKHLTPVTLELGGKSPCYVDKNCDLDVACRRIAWGKFMNSGQTCVAPDYILCDPSIQNQIVEKLKKSLKEFYGEDAKKSRDYGRIISARHFQRVMGLIEGQKVAYGGTGDAATRYIAPTILTDVDPQSPVMQEEIFGPVLPIVCVRSLEEAIQFINQREKPLALYMFSSNDKVIKKMIAETSSGGVAANDVIVHITLHSLPFGGVGNSGMGSYHGKKSFETFSHRRSCLVRPLMNDEGLKVRYPPSPAKMTQH",
# S10A7_HUMAN
"MSNTQAERSIIGMIDMFHKYTRRDDKIEKPSLLTMMKENFPNFLSACDKKGTNYLADVFEKKDKNEDKKIDFSEFLSLLGDIATDYHKQSHGAAPCSGGSQ",
# MA1A1_HUMAN
"MPVGGLLPLFSSPAGGVLGGGLGGGGGRKGSGPAALRLTEKFVLLLVFSAFITLCFGAIFFLPDSSKLLSGVLFHSSPALQPAADHKPGPGARAEDAAEGRARRREEGAPGDPEAALEDNLARIRENHERALREAKETLQKLPEEIQRDILLEKKKVAQDQLRDKAPFRGLPPVDFVPPIGVESREPADAAIREKRAKIKEMMKHAWNNYKGYAWGLNELKPISKGGHSSSLFGNIKGATIVDALDTLFIMEMKHEFEEAKSWVEENLDFNVNAEISVFEVNIRFVGGLLSAYYLSGEEIFRKKAVELGVKLLPAFHTPSGIPWALLNMKSGIGRNWPWASGGSSILAEFGTLHLEFMHLSHLSGNPIFAEKVMNIRTVLNKLEKPQGLYPNYLNPSSGQWGQHHVSVGGLGDSFYEYLLKAWLMSDKTDLEAKKMYFDAVQAIETHLIRKSSSGLTYIAEWKGGLLEHKMGHLTCFAGGMFALGADAAPEGMAQHYLELGAEIARTCHESYNRTFMKLGPEAFRFDGGVEAIATRQNEKYYILRPEVMETYMYMWRLTHDPKYRKWAWEAVEALENHCRVNGGYSGLRDVYLLHESYDDVQQSFFLAETLKYLYLIFSDDDLLPLEHWIFNSEAHLLPILPKDKKEVEIREE",
# MP2K2_HUMAN
"MLARRKPVLPALTINPTIAEGPSPTSEGASEANLVDLQKKLEELELDEQQKKRLEAFLTQKAKVGELKDDDFERISELGAGNGGVVTKVQHRPSGLIMARKLIHLEIKPAIRNQIIRELQVLHECNSPYIVGFYGAFYSDGEISICMEHMDGGSLDQVLKEAKRIPEEILGKVSIAVLRGLAYLREKHQIMHRDVKPSNILVNSRGEIKLCDFGVSGQLIDSMANSFVGTRSYMAPERLQGTHYSVQSDIWSMGLSLVELAVGRYPIPPPDAKELEAIFGRPVVDGEEGEPHSISPRPRPPGRPVSGHGMDSRPAMAIFELLDYIVNEPPPKLPNGVFTPDFQEFVNKCLIKNPAERADLKMLTNHTFIKRSEVEEVDFAGWLCKTLRLNQPGTPTRTAV",
# VHL_HUMAN
"MPRRAENWDEAEVGAEEAGVEEYGPEEDGGEESGAEESGPEESGPEELGAEEEMEAGRPRPVLRSVNSREPSQVIFCNRSPRVVLPVWLNFDGEPQPYPTLPPGTGRRIHSYRGHLWLFRDAGTHDGLLVNQTELFVPSLNVDGQPIFANITLPVYTLKERCLQVVRSLVKPENYRRLDIVRSLYEDLEDHPNVQKDLERLTQERIAHQRMGD",
# HD_HUMAN
"MATLEKLMKAFESLKSFQQQQQQQQQQQQQQQQQQQQQPPPPPPPPPPPQLPQPPPQAQPLLPQPQPPPPPPPPPPGPAVAEEPLHRPKKELSATKKDRVNHCLTICENIVAQSVRNSPEFQKLLGIAMELFLLCSDDAESDVRMVADECLNKVIKALMDSNLPRLQLELYKEIKKNGAPRSLRAALWRFAELAHLVRPQKCRPYLVNLLPCLTRTSKRPEESVQETLAAAVPKIMASFGNFANDNEIKVLLKAFIANLKSSSPTIRRTAAGSAVSICQHSRRTQYFYSWLLNVLLGLLVPVEDEHSTLLILGVLLTLRYLVPLLQQQVKDTSLKGSFGVTRKEMEVSPSAEQLVQVYELTLHHTQHQDHNVVTGALELLQQLFRTPPPELLQTLTAVGGIGQLTAAKEESGGRSRSGSIVELIAGGGSSCSPVLSRKQKGKVLLGEEEALEDDSESRSDVSSSALTASVKDEISGELAASSGVSTPGSAGHDIITEQPRSQHTLQADSVDLASCDLTSSATDGDEEDILSHSSSQVSAVPSDPAMDLNDGTQASSPISDSSQTTTEGPDSAVTPSDSSEIVLDGTDNQYLGLQIGQPQDEDEEATGILPDEASEAFRNSSMALQQAHLLKNMSHCRQPSDSSVDKFVLRDEATEPGDQENKPCRIKGDIGQSTDDDSAPLVHCVRLLSASFLLTGGKNVLVPDRDVRVSVKALALSCVGAAVALHPESFFSKLYKVPLDTTEYPEEQYVSDILNYIDHGDPQVRGATAILCGTLICSILSRSRFHVGDWMGTIRTLTGNTFSLADCIPLLRKTLKDESSVTCKLACTAVRNCVMSLCSSSYSELGLQLIIDVLTLRNSSYWLVRTELLETLAEIDFRLVSFLEAKAENLHRGAHHYTGLLKLQERVLNNVVIHLLGDEDPRVRHVAAASLIRLVPKLFYKCDQGQADPVVAVARDQSSVYLKLLMHETQPPSHFSVSTITRIYRGYNLLPSITDVTMENNLSRVIAAVSHELITSTTRALTFGCCEALCLLSTAFPVCIWSLGWHCGVPPLSASDESRKSCTVGMATMILTLLSSAWFPLDLSAHQDALILAGNLLAASAPKSLRSSWASEEEANPAATKQEEVWPALGDRALVPMVEQLFSHLLKVINICAHVLDDVAPGPAIKAALPSLTNPPSLSPIRRKGKEKEPGEQASVPLSPKKGSEASAASRQSDTSGPVTTSKSSSLGSFYHLPSYLKLHDVLKATHANYKVTLDLQNSTEKFGGFLRSALDVLSQILELATLQDIGKCVEEILGYLKSCFSREPMMATVCVQQLLKTLFGTNLASQFDGLSSNPSKSQGRAQRLGSSSVRPGLYHYCFMAPYTHFTQALADASLRNMVQAEQENDTSGWFDVLQKVSTQLKTNLTSVTKNRADKNAIHNHIRLFEPLVIKALKQYTTTTCVQLQKQVLDLLAQLVQLRVNYCLLDSDQVFIGFVLKQFEYIEVGQFRESEAIIPNIFFFLVLLSYERYHSKQIIGIPKIIQLCDGIMASGRKAVTHAIPALQPIVHDLFVLRGTNKADAGKELETQKEVVVSMLLRLIQYHQVLEMFILVLQQCHKENEDKWKRLSRQIADIILPMLAKQQMHIDSHEALGVLNTLFEILAPSSLRPVDMLLRSMFVTPNTMASVSTVQLWISGILAILRVLISQSTEDIVLSRIQELSFSPYLISCTVINRLRDGDSTSTLEEHSEGKQIKNLPEETFSRFLLQLVGILLEDIVTKQLKVEMSEQQHTFYCQELGTLLMCLIHIFKSGMFRRITAAATRLFRSDGCGGSFYTLDSLNLRARSMITTHPALVLLWCQILLLVNHTDYRWWAEVQQTPKRHSLSSTKLLSPQMSGEEEDSDLAAKLGMCNREIVRRGALILFCDYVCQNLHDSEHLTWLIVNHIQDLISLSHEPPVQDFISAVHRNSAASGLFIQAIQSRCENLSTPTMLKKTLQCLEGIHLSQSGAVLTLYVDRLLCTPFRVLARMVDILACRRVEMLLAANLQSSMAQLPMEELNRIQEYLQSSGLAQRHQRLYSLLDRFRLSTMQDSLSPSPPVSSHPLDGDGHVSLETVSPDKDWYVHLVKSQCWTRSDSALLEGAELVNRIPAEDMNAFMMNSEFNLSLLAPCLSLGMSEISGGQKSALFEAAREVTLARVSGTVQQLPAVHHVFQPELPAEPAAYWSKLNDLFGDAALYQSLPTLARALAQYLVVVSKLPSHLHLPPEKEKDIVKFVVATLEALSWHLIHEQIPLSLDLQAGLDCCCLALQLPGLWSVVSSTEFVTHACSLIYCVHFILEAVAVQPGEQLLSPERRTNTPKAISEEEEEVDPNTQNPKYITAACEMVAEMVESLQSVLALGHKRNSGVPAFLTPLLRNIIISLARLPLVNSYTRVPPLVWKLGWSPKPGGDFGTAFPEIPVEFLQEKEVFKEFIYRINTLGWTSRTQFEETWATLLGVLVTQPLVMEQEESPPEEDTERTQINVLAVQAITSLVLSAMTVPVAGNPAVSCLEQQPRNKPLKALDTRFGRKLSIIRGIVEQEIQAMVSKRENIATHHLYQAWDPVPSLSPATTGALISHEKLLLQINPERELGSMSYKLGQVSIHSVWLGNSITPLREEEWDEEEEEEADAPAPSSPPTSPVNSRKHRAGVDIHSCSQFLLELYSRWILPSSSARRTPAILISEVVRSLLVVSDLFTERNQFELMYVTLTELRRVHPSEDEILAQYLVPATCKAAAVLGMDKAVAEPVSRLLESTLRSSHLPSRVGALHGVLYVLECDLLDDTAKQLIPVISDYLLSNLKGIAHCVNIHSQQHVLVMCATAFYLIENYPLDVGPEFSASIIQMCGVMLSGSEESTPSIIYHCALRGLERLLLSEQLSRLDAESLVKLSVDRVNVHSPHRAMAALGLMLTCMYTGKEKVSPGRTSDPNPAAPDSESVIVAMERVSVLFDRIRKGFPCEARVVARILPQFLDDFFPPQDIMNKVIGEFLSNQQPYPQFMATVVYKVFQTLHSTGQSSMVRDWVMLSLSNFTQRAPVAMATWSLSCFFVSASTSPWVAAILPHVISRMGKLEQVDVNLFCLVATDFYRHQIEEELDRRAFQSVLEVVAAPGSPYHRLLTCLRNVHKVTTC",
# CACP_HUMAN
"MLAFAARTVVKPLGFLKPFSLMKASSRFKAHQDALPRLPVPPLQQSLDHYLKALQPIVSEEEWAHTKQLVDEFQASGGVGERLQKGLERRARKTENWLSEWWLKTAYLQYRQPVVIYSSPGVMLPKQDFVDLQGQLRFAAKLIEGVLDFKVMIDNETLPVEYLGGKPLCMNQYYQILSSCRVPGPKQDTVSNFSKTKKPPTHITVVHNYQFFELDVYHSDGTPLTADQIFVQLEKIWNSSLQTNKEPVGILTSNHRNSWAKAYNTLIKDKVNRDSVRSIQKSIFTVCLDATMPRVSEDVYRSHVAGQMLHGGGSRLNSGNRWFDKTLQFIVAEDGSCGLVYEHAAAEGPPIVTLLDYVIEYTKKPELVRSPLVPLPMPKKLRFNITPEIKSDIEKAKQNLSIMIQDLDITVMVFHHFGKDFPKSEKLSPDAFIQMALQLAYYRIYGQACATYESASLRMFHLGRTDTIRSASMDSLTFVKAMDDSSVTEHQKVELLRKAVQAHRGYTDRAIRGEAFDRHLLGLKLQAIEDLVSMPDIFMDTSYAIAMHFHLSTSQVPAKTDCVMFFGPVVPDGYGVCYNPMEAHINFSLSAYNSCAETNAARLAHYLEKALLDMRALLQSHPRAKL",
# RL27A_HUMAN
"MPSRLRKTRKLRGHVSHGHGRIGKHRKHPGGRGNAGGLHHHRINFDKYHPGYFGKVGMKHYHLKRNQSFCPTVNLDKLWTLVSEQTRVNAAKNKTGAAPIIDVVRSGYYKVLGKGKLPKQPVIVKAKFFSRRAEEKIKSVGGACVLVA",
# HSP13_HUMAN
"MAREMTILGSAVLTLLLAGYLAQQYLPLPTPKVIGIDLGTTYCSVGVFFPGTGKVKVIPDENGHISIPSMVSFTDNDVYVGYESVELADSNPQNTIYDAKRFIGKIFTAEELEAEIGRYPFKVLNKNGMVEFSVTSNETITVSPEYVGSRLLLKLKEMAEAYLGMPVANAVISVPAEFDLKQRNSTIEAANLAGLKILRVINEPTAAAMAYGLHKADVFHVLVIDLGGGTLDVSLLNKQGGMFLTRAMSGNNKLGGQDFNQRLLQYLYKQIYQTYGFVPSRKEEIHRLRQAVEMVKLNLTLHQSAQLSVLLTVEEQDRKEPHSSDTELPKDKLSSADDHRVNSGFGRGLSDKKSGESQVLFETEISRKLFDTLNEDLFQKILVPIQQVLKEGHLEKTEIDEVVLVGGSTRIPRIRQVIQEFFGKDPNTSVDPDLAVVTGVAIQAGIDGGSWPLQVSALEIPNKHLQKTNFN",
# EMD_HUMAN
"MDNYADLSDTELTTLLRRYNIPHGPVVGSTRRLYEKKIFEYETQRRRLSPPSSSAASSYSFSDLNSTRGDADMYDLPKKEDALLYQSKGYNDDYYEESYFTTRTYGEPESAGPSRAVRQSVTSFPDADAFHHQVHDDDLLSSSEEECKDRERPMYGRDSAYQSITHYRPVSASRSSLDLSYYPTSSSTSFMSSSSSSSSWLTRRAIRPENRAPGAGLGQDRQVPLWGQLLLFLVFVIVLFFIYHFMQAEEGNPF",
# GDS1_HUMAN
"MDNLSDTLKKLKITAVDKTEDSLEGCLDCLLQALAQNNTETSEKIQASGILQLFASLLTPQSSCKAKVANIIAEVAKNEFMRIPCVDAGLISPLVQLLNSKDQEVLLQTGRALGNICYDSHEGRSAVDQAGGAQIVIDHLRSLCSITDPANEKLLTVFCGMLMNYSNENDSLQAQLINMGVIPTLVKLLGIHCQNAALTEMCLVAFGNLAELESSKEQFASTNIAEELVKLFKKQIEHDKREMIFEVLAPLAENDAIKLQLVEAGLVECLLEIVQQKVDSDKEDDITELKTGSDLMVLLLLGDESMQKLFEGGKGSVFQRVLSWIPSNNHQLQLAGALAIANFARNDANCIHMVDNGIVEKLMDLLDRHVEDGNVTVQHAALSALRNLAIPVINKAKMLSAGVTEAVLKFLKSEMPPVQFKLLGTLRMLIDAQAEAAEQLGKNVKLVERLVEWCEAKDHAGVMGESNRLLSALIRHSKSKDVIKTIVQSGGIKHLVTMATSEHVIMQNEALVALALIAALELGTAEKDLESAKLVQILHRLLADERSAPEIKYNSMVLICALMGSECLHKEVQDLAFLDVVSKLRSHENKSVAQQASLTEQRLTVES",
# HAP1_HUMAN
"MRPKRLGRCCAGSRLGPGDPAALTCAPSPSASPAPEPSAQPQARGTGQRVGSRATSGSQFLSEARTGARPASEAGAKAGARRPSAFSAIQGDVRSMPDNSDAPWTRFVFQGPFGSRATGRGTGKAAGIWKTPAAYVGRRPGVSGPERAAFIRELEEALCPNLPPPVKKITQEDVKVMLYLLEELLPPVWESVTYGMVLQRERDLNTAARIGQSLVKQNSVLMEENSKLEALLGSAKEEILYLRHQVNLRDELLQLYSDSDEEDEDEEEEEEEKEAEEEQEEEEAEEDLQCAHPCDAPKLISQEALLHQHHCPQLEALQEKLRLLEEENHQLREEASQLDTLEDEEQMLILECVEQFSEASQQMAELSEVLVLRLENYERQQQEVARLQAQVLKLQQRCRMYGAETEKLQKQLASEKEIQMQLQEESVWVGSQLQDLREKYMDCGGMLIEMQEEVKTLRQQPPVSTGSATHYPYSVPLETLPGFQETLAEELRTSLRRMISDPVYFMERNYEMPRGDTSSLRYDFRYSEDREQVRGFEAEEGLMLAADIMRGEDFTPAEEFVPQEELGAAKKVPAEEGVMEEAELVSEETEGWEEVELELDEATRMNVVTSALEASGLGPSHLDMNYVLQQLANWQDAHYRRQLRWKMLQKGECPHGALPAASRTSCRSSCR",
# EPHB1_HUMAN
"MALDYLLLLLLASAVAAMEETLMDTRTATAELGWTANPASGWEEVSGYDENLNTIRTYQVCNVFEPNQNNWLLTTFINRRGAHRIYTEMRFTVRDCSSLPNVPGSCKETFNLYYYETDSVIATKKSAFWSEAPYLKVDTIAADESFSQVDFGGRLMKVNTEVRSFGPLTRNGFYLAFQDYGACMSLLSVRVFFKKCPSIVQNFAVFPETMTGAESTSLVIARGTCIPNAEEVDVPIKLYCNGDGEWMVPIGRCTCKPGYEPENSVACKACPAGTFKASQEAEGCSHCPSNSRSPAEASPICTCRTGYYRADFDPPEVACTSVPSGPRNVISIVNETSIILEWHPPRETGGRDDVTYNIICKKCRADRRSCSRCDDNVEFVPRQLGLTECRVSISSLWAHTPYTFDIQAINGVSSKSPFPPQHVSVNITTNQAAPSTVPIMHQVSATMRSITLSWPQPEQPNGIILDYEIRYYEKEHNEFNSSMARSQTNTARIDGLRPGMVYVVQVRARTVAGYGKFSGKMCFQTLTDDDYKSELREQLPLIAGSAAAGVVFVVSLVAISIVCSRKRAYSKEAVYSDKLQHYSTGRGSPGMKIYIDPFTYEDPNEAVREFAKEIDVSFVKIEEVIGAGEFGEVYKGRLKLPGKREIYVAIKTLKAGYSEKQRRDFLSEASIMGQFDHPNIIRLEGVVTKSRPVMIITEFMENGALDSFLRQNDGQFTVIQLVGMLRGIAAGMKYLAEMNYVHRDLAARNILVNSNLVCKVSDFGLSRYLQDDTSDPTYTSSLGGKIPVRWTAPEAIAYRKFTSASDVWSYGIVMWEVMSFGERPYWDMSNQDVINAIEQDYRLPPPMDCPAALHQLMLDCWQKDRNSRPRFAEIVNTLDKMIRNPASLKTVATITAVPSQPLLDRSIPDFTAFTTVDDWLSAIKMVQYRDSFLTAGFTSLQLVTQMTSEDLLRIGITLAGHQKKILNSIHSMRVQISQSPTAMA",
# TCL1A_HUMAN
"MAECPTLGEAVTDHPDRLWAWEKFVYLDEKQHAWLPLTIEIKDRLQLRVLLRREDVVLGRPMTPTQIGPSLLPIMWQLYPDGRYRSSDSSFWRLVYHIKIDGVEDMLLELLPDD",
# RAB38_HUMAN
"MQAPHKEHLYKLLVIGDLGVGKTSIIKRYVHQNFSSHYRATIGVDFALKVLHWDPETVVRLQLWDIAGQERFGNMTRVYYREAMGAFIVFDVTRPATFEAVAKWKNDLDSKLSLPNGKPVSVVLLANKCDQGKDVLMNNGLKMDQFCKEHGFVGWFETSAKENINIDEASRCLVKHILANECDLMESIEPDVVKPHLTSTKVASCSGCAKS",
# UB2G2_HUMAN
"MAGTALKRLMAEYKQLTLNPPEGIVAGPMNEENFFEWEALIMGPEDTCFEFGVFPAILSFPLDYPLSPPKMRFTCEMFHPNIYPDGRVCISILHAPGDDPMGYESSAERWSPVQSVEKILLSVVSMLAEPNDESGANVDASKMWRDDREQFYKIAKQIVQKSLGL",
# RS3A_HUMAN
"MAVGKNKRLTKGGKKGAKKKVVDPFSKKDWYDVKAPAMFNIRNIGKTLVTRTQGTKIASDGLKGRVFEVSLADLQNDEVAFRKFKLITEDVQGKNCLTNFHGMDLTRDKMCSMVKKWQTMIEAHVDVKTTDGYLLRLFCVGFTKKRNNQIRKTSYAQHQQVRQIRKKMMEIMTREVQTNDLKEVVNKLIPDSIGKDIEKACQSIYPLHDVFVRKVKMLKKPKFELGKLMELHGEGSSSGKATGDETGAKVERADGYEPPVQESV",
# RS8_HUMAN
"MGISRDNWHKRRKTGGKRKPYHKKRKYELGRPAANTKIGPRRIHTVRVRGGNKKYRALRLDVGNFSWGSECCTRKTRIIDVVYNASNNELVRTKTLVKNCIVLIDSTPYRQWYESHYALPLGRKKGAKLTPEEEEILNKKRSKKIQKKYDERKKNAKISSLLEEQFQQGKLLACIASRPGQCGRADGYVLEGKELEFYLRKIKARKGK",
# CNBP_HUMAN
"MSSNECFKCGRSGHWARECPTGGGRGRGMRSRGRGGFTSDRGFQFVSSSLPDICYRCGESGHLAKDCDLQEDACYNCGRGGHIAKDCKEPKREREQCCYNCGKPGHLARDCDHADEQKCYSCGEFGHIQKDCTKVKCYRCGETGHVAINCSKTSEVNCYRCGESGHLARECTIEATA",
# UBX2A_HUMAN
"MKDVDNLKSIKEEWVCETGSDNQPLGNNQQSNCEYFVDSLFEEAQKVSSKCVSPAEQKKQVDVNIKLWKNGFTVNDDFRSYSDGASQQFLNSIKKGELPSELQGIFDKEEVDVKVEDKKNEICLSTKPVFQPFSGQGHRLGSATPKIVSKAKNIEVENKNNLSAVPLNNLEPITNIQIWLANGKRIVQKFNITHRVSHIKDFIEKYQGSQRSPPFSLATALPVLRLLDETLTLEEADLQNAVIIQRLQKTASFRELSEH",
# DLG4_HUMAN
"MDCLCIVTTKKYRYQDEDTPPLEHSPAHLPNQANSPPVIVNTDTLEAPGYELQVNGTEGEMEYEEITLERGNSGLGFSIAGGTDNPHIGDDPSIFITKIIPGGAAAQDGRLRVNDSILFVNEVDVREVTHSAAVEALKEAGSIVRLYVMRRKPPAEKVMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGRLQIGDKILAVNSVGLEDVMHEDAVAALKNTYDVVYLKVAKPSNAYLSDSYAPPDITTSYSQHLDNEISHSSYLGTDYPTAMTPTSPRRYSPVAKDLLGEEDIPREPRRIVIHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDQILSVNGVDLRNASHEQAAIALKNAGQTVTIIAQYKPEEYSRFEAKIHDLREQLMNSSLGSGTASLRSNPKRGFYIRALFDYDKTKDCGFLSQALSFRFGDVLHVIDASDEEWWQARRVHSDSETDDIGFIPSKRRVERREWSRLKAKDWGSSSGSQGREDSVLSYETVTQMEVHYARPIIILGPTKDRANDDLLSEFPDKFGSCVPHTTRPKREYEIDGRDYHFVSSREKMEKDIQAHKFIEAGQYNSHLYGTSVQSVREVAEQGKHCILDVSANAVRRLQAAHLHPIAIFIRPRSLENVLEINKRITEEQARKAFDRATKLEQEFTECFSAIVEGDSFEEIYHKVKRVIEDLSGPYIWVPARERL",
# NUCB2_HUMAN
"MRWRTILLQYCFLLITCLLTALEAVPIDIDKTKVQNIHPVESAKIEPPDTGLYYDEYLKQVIDVLETDKHFREKLQKADIEEIKSGRLSKELDLVSHHVRTKLDELKRQEVGRLRMLIKAKLDSLQDIGMDHQALLKQFDHLNHLNPDKFESTDLDMLIKAATSDLEHYDKTRHEEFKKYEMMKEHERREYLKTLNEEKRKEEESKFEEMKKKHENHPKVNHPGSKDQLKEVWEETDGLDPNDFDPKTFFKLHDVNSDGFLDEQELEALFTKELEKVYDPKNEEDDMVEMEEERLRMREHVMNEVDTNKDRLVTLEEFLKATEKKEFLEPDSWETLDQQQFFTEEELKEYENIIALQENELKKKADELQKQKEELQRQHDQLEAQKLEYHQVIQQMEQKKLQQGIPPSGPAGELKFEPHI",
# TMF1_HUMAN
"MSWFNASQLSSFAKQALSQAQKSIDRVLDIQEEEPSIWAETIPYGEPGISSPVSGGWDTSTWGLKSNTEPQSPPIASPKAITKPVRRTVVDESENFFSAFLSPTDVQTIQKSPVVSKPPAKSQRPEEEVKSSLHESLHIGQSRTPETTESQVKDSSLCVSGETLAAGTSSPKTEGKHEETVNKESDMKVPTVSLKVSESVIDVKTTMESISNTSTQSLTAETKDIALEPKEQKHEDRQSNTPSPPVSTFSSGTSTTSDIEVLDHESVISESSASSRQETTDSKSSLHLMQTSFQLLSASACPEYNRLDDFQKLTESCCSSDAFERIDSFSVQSLDSRSVSEINSDDELSGKGYALVPIIVNSSTPKSKTVESAEGKSEEVNETLVIPTEEAEMEESGRSATPVNCEQPDILVSSTPINEGQTVLDKVAEQCEPAESQPEALSEKEDVCKTVEFLNEKLEKREAQLLSLSKEKALLEEAFDNLKDEMFRVKEESSSISSLKDEFTQRIAEAEKKVQLACKERDAAKKEIKNIKEELATRLNSSETADLLKEKDEQIRGLMEEGEKLSKQQLHNSNIIKKLRAKDKENENMVAKLNKKVKELEEELQHLKQVLDGKEEVEKQHRENIKKLNSMVERQEKDLGRLQVDMDELEEKNRSIQAALDSAYKELTDLHKANAAKDSEAQEAALSREMKAKEELSAALEKAQEEARQQQETLAIQVGDLRLALQRTEQAAARKEDYLRHEIGELQQRLQEAENRNQELSQSVSSTTRPLLRQIENLQATLGSQTSSWEKLEKNLSDRLGESQTLLAAAVERERAATEELLANKIQMSSMESQNSLLRQENSRFQAQLESEKNRLCKLEDENNRYQVELENLKDEYVRTLEETRKEKTLLNSQLEMERMKVEQERKKAIFTQETIKEKERKPFSVSSTPTMSRSSSISGVDMAGLQTSFLSQDESHDHSFGPMPISANGSNLYDAVRMGAGSSIIENLQSQLKLREGEITHLQLEIGNLEKTRSIMAEELVKLTNQNDELEEKVKEIPKLRTQLRDLDQRYNTILQMYGEKAEEAEELRLDLEDVKNMYKTQIDELLRQSLS",
# SET_HUMAN
"MAPKRQSPLPPQKKKPRPPPALGPEETSASAGLPKKGEKEQQEAIEHIDEVQNEIDRLNEQASEEILKVEQKYNKLRQPFFQKRSELIAKIPNFWVTTFVNHPQVSALLGEEDEEALHYLTRVEVTEFEDIKSGYRIDFYFDENPYFENKVLSKEFHLNESGDPSSKSTEIKWKSGKDLTKRSSQTQNKASRKRQHEEPESFFTWFTDHSDAGADELGEVIKDDIWPNPLQYYLVPDMDDEEGEGEEDDDDDEEEEGLEDIDEEGDEDEGEEDEDDDEGEEGEEDEGEDD",
# KPCE_HUMAN
"MVVFNGLLKIKICEAVSLKPTAWSLRHAVGPRPQTFLLDPYIALNVDDSRIGQTATKQKTNSPAWHDEFVTDVCNGRKIELAVFHDAPIGYDDFVANCTIQFEELLQNGSRHFEDWIDLEPEGRVYVIIDLSGSSGEAPKDNEERVFRERMRPRKRQGAVRRRVHQVNGHKFMATYLRQPTYCSHCRDFIWGVIGKQGYQCQVCTCVVHKRCHELIITKCAGLKKQETPDQVGSQRFSVNMPHKFGIHNYKVPTFCDHCGSLLWGLLRQGLQCKVCKMNVHRRCETNVAPNCGVDARGIAKVLADLGVTPDKITNSGQRRKKLIAGAESPQPASGSSPSEEDRSKSAPTSPCDQEIKELENNIRKALSFDNRGEEHRAASSPDGQLMSPGENGEVRQGQAKRLGLDEFNFIKVLGKGSFGKVMLAELKGKDEVYAVKVLKKDVILQDDDVDCTMTEKRILALARKHPYLTQLYCCFQTKDRLFFVMEYVNGGDLMFQIQRSRKFDEPRSRFYAAEVTSALMFLHQHGVIYRDLKLDNILLDAEGHCKLADFGMCKEGILNGVTTTTFCGTPDYIAPEILQELEYGPSVDWWALGVLMYEMMAGQPPFEADNEDDLFESILHDDVLYPVWLSKEAVSILKAFMTKNPHKRLGCVASQNGEDAIKQHPFFKEIDWVLLEQKKIKPPFKPRIKTKRDVNNFDQDFTREEPVLTLVDEAIVKQINQEEFKGFSYFGEDLMP",
# MP2K1_HUMAN
"MPKKKPTPIQLNPAPDGSAVNGTSSAETNLEALQKKLEELELDEQQRKRLEAFLTQKQKVGELKDDDFEKISELGAGNGGVVFKVSHKPSGLVMARKLIHLEIKPAIRNQIIRELQVLHECNSPYIVGFYGAFYSDGEISICMEHMDGGSLDQVLKKAGRIPEQILGKVSIAVIKGLTYLREKHKIMHRDVKPSNILVNSRGEIKLCDFGVSGQLIDSMANSFVGTRSYMSPERLQGTHYSVQSDIWSMGLSLVEMAVGRYPIPPPDAKELELMFGCQVEGDAAETPPRPRTPGRPLSSYGMDSRPPMAIFELLDYIVNEPPPKLPSGVFSLEFQDFVNKCLIKNPAERADLKQLMVHAFIKRSDAEEVDFAGWLCSTIGLNQPSTPTHAAGV",
# ATP7A_HUMAN
"MDPSMGVNSVTISVEGMTCNSCVWTIEQQIGKVNGVHHIKVSLEEKNATIIYDPKLQTPKTLQEAIDDMGFDAVIHNPDPLPVLTDTLFLTVTASLTLPWDHIQSTLLKTKGVTDIKIYPQKRTVAVTIIPSIVNANQIKELVPELSLDTGTLEKKSGACEDHSMAQAGEVVLKMKVEGMTCHSCTSTIEGKIGKLQGVQRIKVSLDNQEATIVYQPHLISVEEMKKQIEAMGFPAFVKKQPKYLKLGAIDVERLKNTPVKSSEGSQQRSPSYTNDSTATFIIDGMHCKSCVSNIESTLSALQYVSSIVVSLENRSAIVKYNASSVTPESLRKAIEAVSPGLYRVSITSEVESTSNSPSSSSLQKIPLNVVSQPLTQETVINIDGMTCNSCVQSIEGVISKKPGVKSIRVSLANSNGTVEYDPLLTSPETLRGAIEDMGFDATLSDTNEPLVVIAQPSSEMPLLTSTNEFYTKGMTPVQDKEEGKNSSKCYIQVTGMTCASCVANIERNLRREEGIYSILVALMAGKAEVRYNPAVIQPPMIAEFIRELGFGATVIENADEGDGVLELVVRGMTCASCVHKIESSLTKHRGILYCSVALATNKAHIKYDPEIIGPRDIIHTIESLGFEASLVKKDRSASHLDHKREIRQWRRSFLVSLFFCIPVMGLMIYMMVMDHHFATLHHNQNMSKEEMINLHSSMFLERQILPGLSVMNLLSFLLCVPVQFFGGWYFYIQAYKALKHKTANMDVLIVLATTIAFAYSLIILLVAMYERAKVNPITFFDTPPMLFVFIALGRWLEHIAKGKTSEALAKLISLQATEATIVTLDSDNILLSEEQVDVELVQRGDIIKVVPGGKFPVDGRVIEGHSMVDESLITGEAMPVAKKPGSTVIAGSINQNGSLLICATHVGADTTLSQIVKLVEEAQTSKAPIQQFADKLSGYFVPFIVFVSIATLLVWIVIGFLNFEIVETYFPGYNRSISRTETIIRFAFQASITVLCIACPCSLGLATPTAVMVGTGVGAQNGILIKGGEPLEMAHKVKVVVFDKTGTITHGTPVVNQVKVLTESNRISHHKILAIVGTAESNSEHPLGTAITKYCKQELDTETLGTCIDFQVVPGCGISCKVTNIEGLLHKNNWNIEDNNIKNASLVQIDASNEQSSTSSSMIIDAQISNALNAQQYKVLIGNREWMIRNGLVINNDVNDFMTEHERKGRTAVLVAVDDELCGLIAIADTVKPEAELAIHILKSMGLEVVLMTGDNSKTARSIASQVGITKVFAEVLPSHKVAKVKQLQEEGKRVAMVGDGINDSPALAMANVGIAIGTGTDVAIEAADVVLIRNDLLDVVASIDLSRKTVKRIRINFVFALIYNLVGIPIAAGVFMPIGLVLQPWMGSAAMAASSVSVVLSSLFLKLYRKPTYESYELPARSQIGQKSPSEISVHVGIDDTSRNSPKLGLLDRIVNYSRASINSLLSDKRSLNSVVTSEPDKHSLLVGDFREDDDTAL",
# KPCD_HUMAN
"MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQKKPTMYPEWKSTFDAHIYEGRVIQIVLMRAAEEPVSEVTVGVSVLAERCKKNNGKAEFWLDLQPQAKVLMSVQYFLEDVDCKQSMRSEDEAKFPTMNRRGAIKQAKIHYIKNHEFIATFFGQPTFCSVCKDFVWGLNKQGYKCRQCNAAIHKKCIDKIIGRCTGTAANSRDTIFQKERFNIDMPHRFKVHNYMSPTFCDHCGSLLWGLVKQGLKCEDCGMNVHHKCREKVANLCGINQKLLAEALNQVTQRASRRSDSASSEPVGIYQGFEKKTGVAGEDMQDNSGTYGKIWEGSSKCNINNFIFHKVLGKGSFGKVLLGELKGRGEYFAIKALKKDVVLIDDDVECTMVEKRVLTLAAENPFLTHLICTFQTKDHLFFVMEFLNGGDLMYHIQDKGRFELYRATFYAAEIMCGLQFLHSKGIIYRDLKLDNVLLDRDGHIKIADFGMCKENIFGESRASTFCGTPDYIAPEILQGLKYTFSVDWWSFGVLLYEMLIGQSPFHGDDEDELFESIRVDTPHYPRWITKESKDILEKLFEREPTKRLGVTGNIKIHPFFKTINWTLLEKRRLEPPFRPKVKSPRDYSNFDQEFLNEKARLSYSDKNLIDSMDQSAFAGFSFVNPKFEHLLED",
# RL18_HUMAN
"MGVDIRHNKDRKVRRKEPKSQDIYLRLLVKLYRFLARRTNSTFNQVVLKRLFMSRTNRPPLSLSRMIRKMKLPGRENKTAVVVGTITDDVRVQEVPKLKVCALRVTSRARSRILRAGGKILTFDQLALDSPKGCGTVLLSGPRKGREVYRHFGKAPGTPHSHTKPYVRSKGRKFERARGRRASRGYKN",
# B2CL1_HUMAN
"MSQSNRELVVDFLSYKLSQKGYSWSQFSDVEENRTEAPEGTESEMETPSAINGNPSWHLADSPAVNGATGHSSSLDAREVIPMAAVKQALREAGDEFELRYRRAFSDLTSQLHITPGTAYQSFEQVVNELFRDGVNWGRIVAFFSFGGALCVESVDKEMQVLVSRIAAWMATYLNDHLEPWIQENGGWDTFVELYGNNAAAESRKGQERFNRWFLTGMTVAGVVLLGSLFSRK",
# VAC14_HUMAN
"MNPEKDFAPLTPNIVRALNDKLYEKRKVAALEIEKLVREFVAQNNTVQIKHVIQTLSQEFALSQHPHSRKGGLIGLAACSIALGKDSGLYLKELIEPVLTCFNDADSRLRYYACEALYNIVKVARGAVLPHFNVLFDGLSKLAADPDPNVKSGSELLDRLLKDIVTESNKFDLVSFIPLLRERIYSNNQYARQFIISWILVLESVPDINLLDYLPEILDGLFQILGDNGKEIRKMCEVVLGEFLKEIKKNPSSVKFAEMANILVIHCQTTDDLIQLTAMCWMREFIQLAGRVMLPYSSGILTAVLPCLAYDDRKKSIKEVANVCNQSLMKLVTPEDDELDELRPGQRQAEPTPDDALPKQEGTASGGPDGSCDSSFSSGISVFTAASTERAPVTLHLDGIVQVLNCHLSDTAIGMMTRIAVLKWLYHLYIKTPRKMFRHTDSLFPILLQTLSDESDEVILKDLEVLAEIASSPAGQTDDPGPLDGPDLQASHSELQVPTPGRAGLLNTSGTKGLECSPSTPTMNSYFYKFMINLLKRFSSERKLLEVRGPFIIRQLCLLLNAENIFHSMADILLREEDLKFASTMVHALNTILLTSTELFQLRNQLKDLKTLESQNLFCCLYRSWCHNPVTTVSLCFLTQNYRHAYDLIQKFGDLEVTVDFLAEVDKLVQLIECPIFTYLRLQLLDVKNNPYLIKALYGLLMLLPQSSAFQLLSHRLQCVPNPELLQTEDSLKAAPKSQKADSPSIDYAELLQHFEKVQNKHLEVRHQRSGRGDHLDRRVVL",
# KCNA1_HUMAN
"MTVMSGENVDEASAAPGHPQDGSYPRQADHDDHECCERVVINISGLRFETQLKTLAQFPNTLLGNPKKRMRYFDPLRNEYFFDRNRPSFDAILYYYQSGGRLRRPVNVPLDMFSEEIKFYELGEEAMEKFREDEGFIKEEERPLPEKEYQRQVWLLFEYPESSGPARVIAIVSVMVILISIVIFCLETLPELKDDKDFTGTVHRIDNTTVIYNSNIFTDPFFIVETLCIIWFSFELVVRFFACPSKTDFFKNIMNFIDIVAIIPYFITLGTEIAEQEGNQKGEQATSLAILRVIRLVRVFRIFKLSRHSKGLQILGQTLKASMRELGLLIFFLFIGVILFSSAVYFAEAEEAESHFSSIPDAFWWAVVSMTTVGYGDMYPVTIGGKIVGSLCAIAGVLTIALPVPVIVSNFNYFYHRETEGEEQAQLLHVSSPNLASDSDLSRRSSSTMSKSEYMEIEEDMNNSIAHYRQVNIRTANCTTANQNCVNKSKLLTDV",
# U17LO_HUMAN
"MEDDSLYLRGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSCETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYVNASLQCLTYTPPLANYMLSREHSQTCHRHKGCMLCTMQAHITRALHNPGHVIQPSQALAAGFHRGKQEDAHEFLMFTVDAMKKACLPGHKQVDHHSKDTTLIHQIFGGYWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVQQALEQLVKPEELNGENAYHCGVCLQRAPASKTLTLHTSAKVLILVLKRFSDVTGNKIAKNVQYPECLDMQPYMSQPNTGPLVYVLYAVLVHAGWSCHNGHYFSYVKAQEGQWYKMDDAEVTASSITSVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRATQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVRKVEGTLPPDVLVIHQSKYKCGMKNHHPEQQSSLLNLSSSTPTHQESMNTGTLASLRGRARRSKGKNKHSKRALLVCQ",
# WASC5_HUMAN
"MLDFLAENNLCGQAILRIVSCGNAIIAELLRLSEFIPAVFRLKDRADQQKYGDIIFDFSYFKGPELWESKLDAKPELQDLDEEFRENNIEIVTRFYLAFQSVHKYIVDLNRYLDDLNEGVYIQQTLETVLLNEDGKQLLCEALYLYGVMLLVIDQKIEGEVRERMLVSYYRYSAARSSADSNMDDICKLLRSTGYSSQPGAKRPSNYPESYFQRVPINESFISMVIGRLRSDDIYNQVSAYPLPEHRSTALANQAAMLYVILYFEPSILHTHQAKMREIVDKYFPDNWVISIYMGITVNLVDAWEPYKAAKTALNNTLDLSNVREQASRYATVSERVHAQVQQFLKEGYLREEMVLDNIPKLLNCLRDCNVAIRWLMLHTADSACDPNNKRLRQIKDQILTDSRYNPRILFQLLLDTAQFEFILKEMFKQMLSEKQTKWEHYKKEGSERMTELADVFSGVKPLTRVEKNENLQAWFREISKQILSLNYDDSTAAGRKTVQLIQALEEVQEFHQLESNLQVCQFLADTRKFLHQMIRTINIKEEVLITMQIVGDLSFAWQLIDSFTSIMQESIRVNPSMVTKLRATFLKLASALDLPLLRINQANSPDLLSVSQYYSGELVSYVRKVLQIIPESMFTSLLKIIKLQTHDIIEVPTRLDKDKLRDYAQLGPRYEVAKLTHAISIFTEGILMMKTTLVGIIKVDPKQLLEDGIRKELVKRVAFALHRGLIFNPRAKPSELMPKLKELGATMDGFHRSFEYIQDYVNIYGLKIWQEEVSRIINYNVEQECNNFLRTKIQDWQSMYQSTHIPIPKFTPVDESVTFIGRLCREILRITDPKMTCHIDQLNTWYDMKTHQEVTSSRLFSEIQTTLGTFGLNGLDRLLCFMIVKELQNFLSMFQKIILRDRTVQDTLKTLMNAVSPLKSIVANSNKIYFSAIAKTQKIWTAYLEAIMKVGQMQILRQQIANELNYSCRFDSKHLAAALENLNKALLADIEAHYQDPSLPYPKEDNTLLYEITAYLEAAGIHNPLNKIYITTKRLPYFPIVNFLFLIAQLPKLQYNKNLGMVCRKPTDPVDWPPLVLGLLTLLKQFHSRYTEQFLALIGQFICSTVEQCTSQKIPEIPADVVGALLFLEDYVRYTKLPRRVAEAHVPNFIFDEFRTVL",
# HYAL2_HUMAN
"MRAGPGPTVTLALVLAVSWAMELKPTAPPIFTGRPFVVAWDVPTQDCGPRLKVPLDLNAFDVQASPNEGFVNQNITIFYRDRLGLYPRFDSAGRSVHGGVPQNVSLWAHRKMLQKRVEHYIRTQESAGLAVIDWEDWRPVWVRNWQDKDVYRRLSRQLVASRHPDWPPDRIVKQAQYEFEFAAQQFMLETLRYVKAVRPRHLWGFYLFPDCYNHDYVQNWESYTGRCPDVEVARNDQLAWLWAESTALFPSVYLDETLASSRHGRNFVSFRVQEALRVARTHHANHALPVYVFTRPTYSRRLTGLSEMDLISTIGESAALGAAGVILWGDAGYTTSTETCQYLKDYLTRLLVPYVVNVSWATQYCSRAQCHGHGRCVRRNPSASTFLHLSTNSFRLVPGHAPGEPQLRPVGELSWADIDHLQTHFRCQCYLGWSGEQCQWDHRQAAGGASEAWAGSHLTSLLALAALAFTWTL",
# AIMP1_HUMAN
"MANNDAVLKRLEQKGAEADQIIEYLKQQVSLLKEKAILQATLREEKKLRVENAKLKKEIEELKQELIQAEIQNGVKQIPFPSGTPLHANSMVSENVIQSTAVTTVSSGTKEQIKGGTGDEKKAKEKIEKKGEKKEKKQQSIAGSADSKPIDVSRLDLRIGCIITARKHPDADSLYVEEVDVGEIAPRTVVSGLVNHVPLEQMQNRMVILLCNLKPAKMRGVLSQAMVMCASSPEKIEILAPPNGSVPGDRITFDAFPGEPDKELNPKKKIWEQIQPDLHTNDECVATYKGVPFEVKGKGVCRAQTMSNSGIK",
# ANK3_HUMAN
"MAHAASQLKKNRDLEINAEEEPEKKRKHRKRSRDRKKKSDANASYLRAARAGHLEKALDYIKNGVDINICNQNGLNALHLASKEGHVEVVSELLQREANVDAATKKGNTALHIASLAGQAEVVKVLVTNGANVNAQSQNGFTPLYMAAQENHLEVVKFLLDNGASQSLATEDGFTPLAVALQQGHDQVVSLLLENDTKGKVRLPALHIAARKDDTKAAALLLQNDNNADVESKSGFTPLHIAAHYGNINVATLLLNRAAAVDFTARNDITPLHVASKRGNANMVKLLLDRGAKIDAKTRDGLTPLHCGARSGHEQVVEMLLDRAAPILSKTKNGLSPLHMATQGDHLNCVQLLLQHNVPVDDVTNDYLTALHVAAHCGHYKVAKVLLDKKANPNAKALNGFTPLHIACKKNRIKVMELLLKHGASIQAVTESGLTPIHVAAFMGHVNIVSQLMHHGASPNTTNVRGETALHMAARSGQAEVVRYLVQDGAQVEAKAKDDQTPLHISARLGKADIVQQLLQQGASPNAATTSGYTPLHLSAREGHEDVAAFLLDHGASLSITTKKGFTPLHVAAKYGKLEVANLLLQKSASPDAAGKSGLTPLHVAAHYDNQKVALLLLDQGASPHAAAKNGYTPLHIAAKKNQMDIATTLLEYGADANAVTRQGIASVHLAAQEGHVDMVSLLLGRNANVNLSNKSGLTPLHLAAQEDRVNVAEVLVNQGAHVDAQTKMGYTPLHVGCHYGNIKIVNFLLQHSAKVNAKTKNGYTPLHQAAQQGHTHIINVLLQNNASPNELTVNGNTALGIARRLGYISVVDTLKIVTEETMTTTTVTEKHKMNVPETMNEVLDMSDDEVRKANAPEMLSDGEYISDVEEGEDAMTGDTDKYLGPQDLKELGDDSLPAEGYMGFSLGARSASLRSFSSDRSYTLNRSSYARDSMMIEELLVPSKEQHLTFTREFDSDSLRHYSWAADTLDNVNLVSSPIHSGFLVSFMVDARGGSMRGSRHHGMRIIIPPRKCTAPTRITCRLVKRHKLANPPPMVEGEGLASRLVEMGPAGAQFLGPVIVEIPHFGSMRGKERELIVLRSENGETWKEHQFDSKNEDLTELLNGMDEELDSPEELGKKRICRIITKDFPQYFAVVSRIKQESNQIGPEGGILSSTTVPLVQASFPEGALTKRIRVGLQAQPVPDEIVKKILGNKATFSPIVTVEPRRRKFHKPITMTIPVPPPSGEGVSNGYKGDTTPNLRLLCSITGGTSPAQWEDITGTTPLTFIKDCVSFTTNVSARFWLADCHQVLETVGLATQLYRELICVPYMAKFVVFAKMNDPVESSLRCFCMTDDKVDKTLEQQENFEEVARSKDIEVLEGKPIYVDCYGNLAPLTKGGQQLVFNFYSFKENRLPFSIKIRDTSQEPCGRLSFLKEPKTTKGLPQTAVCNLNITLPAHKKETESDQDDEIEKTDRRQSFASLALRKRYSYLTEPGMIERSTGATRSLPTTYSYKPFFSTRPYQSWTTAPITVPGPAKSGFTSLSSSSSNTPSASPLKSIWSVSTPSPIKSTLGASTTSSVKSISDVASPIRSFRTMSSPIKTVVSQSPYNIQVSSGTLARAPAVTEATPLKGLASNSTFSSRTSPVTTAGSLLERSSITMTPPASPKSNINMYSSSLPFKSIITSAAPLISSPLKSVVSPVKSAVDVISSAKITMASSLSSPVKQMPGHAEVALVNGSISPLKYPSSSTLINGCKATATLQEKISSATNSVSSVVSAATDTVEKVFSTTTAMPFSPLRSYVSAAPSAFQSLRTPSASALYTSLGSSISATTSSVTSSIITVPVYSVVNVLPEPALKKLPDSNSFTKSAAALLSPIKTLTTETHPQPHFSRTSSPVKSSLFLAPSALKLSTPSSLSSSQEILKDVAEMKEDLMRMTAILQTDVPEEKPFQPELPKEGRIDDEEPFKIVEKVKEDLVKVSEILKKDVCVDNKGSPKSPKSDKGHSPEDDWIEFSSEEIREARQQAAASQSPSLPERVQVKAKAASEKDYNLTKVIDYLTNDIGSSSLTNLKYKFEDAKKDGEERQKRVLKPAIALQEHKLKMPPASMRTSTSEKELCKMADSFFGTDTILESPDDFSQHDQDKSPLSDSGFETRSEKTPSAPQSAESTGPKPLFHEVPIPPVITETRTEVVHVIRSYDPSAGDVPQTQPEEPVSPKPSPTFMELEPKPTTSSIKEKVKAFQMKASSEEDDHNRVLSKGMRVKEETHITTTTRMVYHSPPGGEGASERIEETMSVHDIMKAFQSGRDPSKELAGLFEHKSAVSPDVHKSAAETSAQHAEKDNQMKPKLERIIEVHIEKGNQAEPTEVIIRETKKHPEKEMYVYQKDLSRGDINLKDFLPEKHDAFPCSEEQGQQEEEELTAEESLPSYLESSRVNTPVSQEEDSRPSSAQLISDDSYKTLKLLSQHSIEYHDDELSELRGESYRFAEKMLLSEKLDVSHSDTEESVTDHAGPPSSELQGSDKRSREKIATAPKKEILSKIYKDVSENGVGKVSKDEHFDKVTVLHYSGNVSSPKHAMWMRFTEDRLDRGREKLIYEDRVDRTVKEAEEKLTEVSQFFRDKTEKLNDELQSPEKKARPKNGKEYSSQSPTSSSPEKVLLTELLASNDEWVKARQHGPDGQGFPKAEEKAPSLPSSPEKMVLSQQTEDSKSTVEAKGSISQSKAPDGPQSGFQLKQSKLSSIRLKFEQGTHAKSKDMSQEDRKSDGQSRIPVKKIQESKLPVYQVFAREKQQKAIDLPDESVSVQKDFMVLKTKDEHAQSNEIVVNDSGSDNVKKQRTEMSSKAMPDSFSEQQAKDLACHITSDLATRGPWDKKVFRTWESSGATNNKSQKEKLSHVLVHDVRENHIGHPESKSVDQKNEFMSVTERERKLLTNGSLSEIKEMTVKSPSKKVLYREYVVKEGDHPGGLLDQPSRRSESSAVSHIPVRVADERRMLSSNIPDGFCEQSAFPKHELSQKLSQSSMSKETVETQHFNSIEDEKVTYSEISKVSKHQSYVGLCPPLEETETSPTKSPDSLEFSPGKESPSSDVFDHSPIDGLEKLAPLAQTEGGKEIKTLPVYVSFVQVGKQYEKEIQQGGVKKIISQECKTVQETRGTFYTTRQQKQPPSPQGSPEDDTLEQVSFLDSSGKSPLTPETPSSEEVSYEFTSKTPDSLIAYIPGKPSPIPEVSEESEEEEQAKSTSLKQTTVEETAVEREMPNDVSKDSNQRPKNNRVAYIEFPPPPPLDADQIESDKKHHYLPEKEVDMIEVNLQDEHDKYQLAEPVIRVQPPSPVPPGADVSDSSDDESIYQPVPVKKYTFKLKEVDDEQKEKPKASAEKASNQKELESNGSGKDNEFGLGLDSPQNEIAQNGNNDQSITECSIATTAEFSHDTDATEIDSLDGYDLQDEDDGLTESDSKLPIQAMEIKKDIWNTEGILKPADRSFSQSKLEVIEEEGKVGPDEDKPPSKSSSSEKTPDKTDQKSGAQFFTLEGRHPDRSVFPDTYFSYKVDEEFATPFKTVATKGLDFDPWSNNRGDDEVFDSKSREDETKPFGLAVEDRSPATTPDTTPARTPTDESTPTSEPNPFPFHEGKMFEMTRSGAIDMSKRDFVEERLQFFQIGEHTSEGKSGDQGEGDKSMVTATPQPQSGDTTVETNLERNVETPTVEPNPSIPTSGECQEGTSSSGSLEKSAAATNTSKVDPKLRTPIKMGISASTMTMKKEGPGEITDKIEAVMTSCQGLENETITMISNTANSQMGVRPHEKHDFQKDNFNNNNNLDSSTIQTDNIMSNIVLTEHSAPTCTTEKDNPVKVSSGKKTGVLQGHCVRDKQKVLGEQQKTKELIGIRQKSKLPIKATSPKDTFPPNHMSNTKASKMKQVSQSEKTKALTTSSCVDVKSRIPVKNTHRDNIIAVRKACATQKQGQPEKGKAKQLPSKLPVKVRSTCVTTTTTTATTTTTTTTTTTTSCTVKVRKSQLKEVCKHSIEYFKGISGETLKLVDRLSEEEKKMQSELSDEEESTSRNTSLSETSRGGQPSVTTKSARDKKTEAAPLKSKSEKAGSEKRSSRRTGPQSPCERTDIRMAIVADHLGLSWTELARELNFSVDEINQIRVENPNSLISQSFMLLKKWVTRDGKNATTDALTSVLTKINRIDIVTLLEGPIFDYGNISGTRSFADENNVFHDPVDGWQNETSSGNLESCAQARRVTGGLLDRLDDSPDQCRDSITSYLKGEAGKFEANGSHTEITPEAKTKSYFPESQNDVGKQSTKETLKPKIHGSGHVEEPASPLAAYQKSLEETSKLIIEETKPCVPVSMKKMSRTSPADGKPRLSLHEEEGSSGSEQKQGEGFKVKTKKEIRHVEKKSHS",
# RHG05_HUMAN
"MMAKNKEPRPPSYTISIVGLSGTEKDKGNCGVGKSCLCNRFVRSKADEYYPEHTSVLSTIDFGGRVVNNDHFLYWGDIIQNSEDGVECKIHVIEQTEFIDDQTFLPHRSTNLQPYIKRAAASKLQSAEKLMYICTDQLGLEQDFEQKQMPEGKLNVDGFLLCIDVSQGCNRKFDDQLKFVNNLFVQLSKSKKPVIIAATKCDECVDHYLREVQAFASNKKNLLVVETSARFNVNIETCFTALVQMLDKTRSKPKIIPYLDAYKTQRQLVVTATDKFEKLVQTVRDYHATWKTVSNKLKNHPDYEEYINLEGTRKARNTFSKHIEQLKQEHIRKRREEYINTLPRAFNTLLPNLEEIEHLNWSEALKLMEKRADFQLCFVVLEKTPWDETDHIDKINDRRIPFDLLSTLEAEKVYQNHVQHLISEKRRVEMKEKFKKTLEKIQFISPGQPWEEVMCFVMEDEAYKYITEADSKEVYGRHQREIVEKAKEEFQEMLFEHSELFYDLDLNATPSSDKMSEIHTVLSEEPRYKALQKLAPDRESLLLKHIGFVYHPTKETCLSGQNCTDIKVEQLLASSLLQLDHGRLRLYHDSTNIDKVNLFILGKDGLAQELANEIRTQSTDDEYALDGKIYELDLRPVDAKSPYFLSQLWTAAFKPHGCFCVFNSIESLSFIGEFIGKIRTEASQIRKDKYMANLPFTLILANQRDSISKNLPILRHQGQQLANKLQCPFVDVPAGTYPRKFNETQIKQALRGVLESVKHNLDVVSPIPANKDLSEADLRIVMCAMCGDPFSVDLILSPFLDSHSCSAAQAGQNNSLMLDKIIGEKRRRIQITILSYHSSIGVRKDELVHGYILVYSAKRKASMGMLRAFLSEVQDTIPVQLVAVTDSQADFFENEAIKELMTEGEHIATEITAKFTALYSLSQYHRQTEVFTLFFSDVLEKKNMIENSYLSDNTRESTHQSEDVFLPSPRDCFPYNNYPDSDDDTEAPPPYSPIGDDVQLLPTPSDRSRYRLDLEGNEYPIHSTPNCHDHERNHKVPPPIKPKPVVPKTNVKKLDPNLLKTIEAGIGKNPRKQTSRVPLAHPEDMDPSDNYAEPIDTIFKQKGYSDEIYVVPDDSQNRIKIRNSFVNNTQGDEENGFSDRTSKSHGERRPSKYKYKSKTLFSKAKSYYRRTHSDASDDEAFTTSKTKRKGRHRGSEEDPLLSPVETWKGGIDNPAITSDQELDDKKMKKKTHKVKEDKKQKKKTKNFNPPTRRNWESNYFGMPLQDLVTAEKPIPLFVEKCVEFIEDTGLCTEGLYRVSGNKTDQDNIQKQFDQDHNINLVSMEVTVNAVAGALKAFFADLPDPLIPYSLHPELLEAAKIPDKTERLHALKEIVKKFHPVNYDVFRYVITHLNRVSQQHKINLMTADNLSICFWPTLMRPDFENREFLSTTKIHQSVVETFIQQCQFFFYNGEIVETTNIVAPPPPSNPGQLVEPMVPLQLPPPLQPQLIQPQLQTDPLGII",
# PDE3B_HUMAN
"MRRDERDAKAMRSLQPPDGAGSPPESLRNGYVKSCVSPLRQDPPRGFFFHLCRFCNVELRPPPASPQQPRRCSPFCRARLSLGALAAFVLALLLGAEPESWAAGAAWLRTLLSVCSHSLSPLFSIACAFFFLTCFLTRTKRGPGPGRSCGSWWLLALPACCYLGDFLVWQWWSWPWGDGDAGSAAPHTPPEAAAGRLLLVLSCVGLLLTLAHPLRLRHCVLVLLLASFVWWVSFTSLGSLPSALRPLLSGLVGGAGCLLALGLDHFFQIREAPLHPRLSSAAEEKVPVIRPRRRSSCVSLGETAASYYGSCKIFRRPSLPCISREQMILWDWDLKQWYKPHYQNSGGGNGVDLSVLNEARNMVSDLLTDPSLPPQVISSLRSISSLMGAFSGSCRPKINPLTPFPGFYPCSEIEDPAEKGDRKLNKGLNRNSLPTPQLRRSSGTSGLLPVEQSSRWDRNNGKRPHQEFGISSQGCYLNGPFNSNLLTIPKQRSSSVSLTHHVGLRRAGVLSSLSPVNSSNHGPVSTGSLTNRSPIEFPDTADFLNKPSVILQRSLGNAPNTPDFYQQLRNSDSNLCNSCGHQMLKYVSTSESDGTDCCSGKSGEEENIFSKESFKLMETQQEEETEKKDSRKLFQEGDKWLTEEAQSEQQTNIEQEVSLDLILVEEYDSLIEKMSNWNFPIFELVEKMGEKSGRILSQVMYTLFQDTGLLEIFKIPTQQFMNYFRALENGYRDIPYHNRIHATDVLHAVWYLTTRPVPGLQQIHNGCGTGNETDSDGRINHGRIAYISSKSCSNPDESYGCLSSNIPALELMALYVAAAMHDYDHPGRTNAFLVATNAPQAVLYNDRSVLENHHAASAWNLYLSRPEYNFLLHLDHVEFKRFRFLVIEAILATDLKKHFDFLAEFNAKANDVNSNGIEWSNENDRLLVCQVCIKLADINGPAKVRDLHLKWTEGIVNEFYEQGDEEANLGLPISPFMDRSSPQLAKLQESFITHIVGPLCNSYDAAGLLPGQWLEAEEDNDTESGDDEDGEELDTEDEEMENNLNPKPPRRKSRRRIFCQLMHHLTENHKIWKEIVEEEEKCKADGNKLQVENSSLPQADEIQVIEEADEEE",
# SQSTM_HUMAN
"MASLTVKAYLLGKEDAAREIRRFSFCCSPEPEAEAEAAAGPGPCERLLSRVAALFPALRPGGFQAHYRDEDGDLVAFSSDEELTMAMSYVKDDIFRIYIKEKKECRRDHRPPCAQEAPRNMVHPNVICDGCNGPVVGTRYKCSVCPDYDLCSVCEGKGLHRGHTKLAFPSPFGHLSEGFSHSRWLRKVKHGHFGWPGWEMGPPGNWSPRPPRAGEARPGPTAESASGPSEDPSVNFLKNVGESVAAALSPLGIEVDIDVEHGGKRSRLTPVSPESSSTEEKSSSQPSSCCSDPSKPGGNVEGATQSLAEQMRKIALESEGRPEEQMESDNCSGGDDDWTHLSSKEVDPSTGELQSLQMPESEGPSSLDPSQEGPTGLKEAALYPHLPPEADPRLIESLSQMLSMGFSDEGGWLTRLLQTKNYDIGAALDTIQYSKHPPPL",
# RAB32_HUMAN
"MAGGGAGDPGLGAAAAPAPETREHLFKVLVIGELGVGKTSIIKRYVHQLFSQHYRATIGVDFALKVLNWDSRTLVRLQLWDIAGQERFGNMTRVYYKEAVGAFVVFDISRSSTFEAVLKWKSDLDSKVHLPNGSPIPAVLLANKCDQNKDSSQSPSQVDQFCKEHGFAGWFETSAKDNINIEEAARFLVEKILVNHQSFPNEENDVDKIKLDQETLRAENKSQCC",
# KEAP1_HUMAN
"MQPDPRPSGAGACCRFLPLQSQCPEGAGDAVMYASTECKAEVTPSQHGNRTFSYTLEDHTKQAFGIMNELRLSQQLCDVTLQVKYQDAPAAQFMAHKVVLASSSPVFKAMFTNGLREQGMEVVSIEGIHPKVMERLIEFAYTASISMGEKCVLHVMNGAVMYQIDSVVRACSDFLVQQLDPSNAIGIANFAEQIGCVELHQRAREYIYMHFGEVAKQEEFFNLSHCQLVTLISRDDLNVRCESEVFHACINWVKYDCEQRRFYVQALLRAVRCHSLTPNFLQMQLQKCEILQSDSRCKDYLVKIFEELTLHKPTQVMPCRAPKVGRLIYTAGGYFRQSLSYLEAYNPSDGTWLRLADLQVPRSGLAGCVVGGLLYAVGGRNNSPDGNTDSSALDCYNPMTNQWSPCAPMSVPRNRIGVGVIDGHIYAVGGSHGCIHHNSVERYEPERDEWHLVAPMLTRRIGVGVAVLNRLLYAVGGFDGTNRLNSAECYYPERNEWRMITAMNTIRSGAGVCVLHNCIYAAGGYDGQDQLNSVERYDVETETWTFVAPMKHRRSALGITVHQGRIYVLGGYDGHTFLDSVECYDPDTDTWSEVTRMTSGRSGVGVAVTMEPCRKQIDQQNCTC",
# UBX2B_HUMAN
"MAEGGGPEPGEQERRSSGPRPPSARDLQLALAELYEDEVKCKSSKSNRPKATVFKSPRTPPQRFYSSEHEYSGLNIVRPSTGKIVNELFKEAREHGAVPLNEATRASGDDKSKSFTGGGYRLGSSFCKRSEYIYGENQLQDVQILLKLWSNGFSLDDGELRPYNEPTNAQFLESVKRGEIPLELQRLVHGGQVNLDMEDHQDQEYIKPRLRFKAFSGEGQKLGSLTPEIVSTPSSPEEEDKSILNAVVLIDDSVPTTKIQIRLADGSRLIQRFNSTHRILDVRNFIVQSRPEFAALDFILVTSFPNKELTDESLTLLEADILNTVLLQQLK",
# MLC1_HUMAN
"MTQEPFREELAYDRMPTLERGRQDPASYAPDAKPSDLQLSKRLPPCFSHKTWVFSVLMGSCLLVTSGFSLYLGNVFPAEMDYLRCAAGSCIPSAIVSFTVSRRNANVIPNFQILFVSTFAVTTTCLIWFGCKLVLNPSAININFNLILLLLLELLMAATVIIAARSSEEDCKKKKGSMSDSANILDEVPFPARVLKSYSVVEVIAGISAVLGGIIALNVDDSVSGPHLSVTFFWILVACFPSAIASHVAAECPSKCLVEVLIAISSLTSPLLFTASGYLSFSIMRIVEMFKDYPPAIKPSYDVLLLLLLLVLLLQAGLNTGTAIQCVRFKVSARLQGASWDTQNGPQERLAGEVARSPLKEFDKEKAWRAVVVQMAQ",
# BAK_HUMAN
"MASGQGPGPPRQECGEPALPSASEEQVAQDTEEVFRSYVFYRHQQEQEAEGVAAPADPEMVTLPLQPSSTMGQVGRQLAIIGDDINRRYDSEFQTMLQHLQPTAENAYEYFTKIATSLFESGINWGRVVALLGFGYRLALHVYQHGLTGFLGQVTRFVVDFMLHHCIARWIAQRGGWVAALNLGNGPILNVLVVLGVVLLGQFVVRRFFKS",
# GRDN_HUMAN
"MENEIFTPLLEQFMTSPLVTWVKTFGPLAAGNGTNLDEYVALVDGVFLNQVMLQINPKLESQRVNKKVNNDASLRMHNLSILVRQIKFYYQETLQQLIMMSLPNVLIIGKNPFSEQGTEEVKKLLLLLLGCAVQCQKKEEFIERIQGLDFDTKAAVAAHIQEVTHNQENVFDLQWMEVTDMSQEDIEPLLKNMALHLKRLIDERDEHSETIIELSEERDGLHFLPHASSSAQSPCGSPGMKRTESRQHLSVELADAKAKIRRLRQELEEKTEQLLDCKQELEQMEIELKRLQQENMNLLSDARSARMYRDELDALREKAVRVDKLESEVSRYKERLHDIEFYKARVEELKEDNQVLLETKTMLEDQLEGTRARSDKLHELEKENLQLKAKLHDMEMERDMDRKKIEELMEENMTLEMAQKQSMDESLHLGWELEQISRTSELSEAPQKSLGHEVNELTSSRLLKLEMENQSLTKTVEELRTTVDSVEGNASKILKMEKENQRLSKKVEILENEIVQEKQSLQNCQNLSKDLMKEKAQLEKTIETLRENSERQIKILEQENEHLNQTVSSLRQRSQISAEARVKDIEKENKILHESIKETSSKLSKIEFEKRQIKKELEHYKEKGERAEELENELHHLEKENELLQKKITNLKITCEKIEALEQENSELERENRKLKKTLDSFKNLTFQLESLEKENSQLDEENLELRRNVESLKCASMKMAQLQLENKELESEKEQLKKGLELLKASFKKTERLEVSYQGLDIENQRLQKTLENSNKKIQQLESELQDLEMENQTLQKNLEELKISSKRLEQLEKENKSLEQETSQLEKDKKQLEKENKRLRQQAEIKDTTLEENNVKIGNLEKENKTLSKEIGIYKESCVRLKELEKENKELVKRATIDIKTLVTLREDLVSEKLKTQQMNNDLEKLTHELEKIGLNKERLLHDEQSTDDSRYKLLESKLESTLKKSLEIKEEKIAALEARLEESTNYNQQLRQELKTVKKNYEALKQRQDEERMVQSSPPISGEDNKWERESQETTRELLKVKDRLIEVERNNATLQAEKQALKTQLKQLETQNNNLQAQILALQRQTVSLQEQNTTLQTQNAKLQVENSTLNSQSTSLMNQNAQLLIQQSSLENENESVIKEREDLKSLYDSLIKDHEKLELLHERQASEYESLISKHGTLKSAHKNLEVEHRDLEDRYNQLLKQKGQLEDLEKMLKVEQEKMLLENKNHETVAAEYKKLCGENDRLNHTYSQLLKETEVLQTDHKNLKSLLNNSKLEQTRLEAEFSKLKEQYQQLDITSTKLNNQCELLSQLKGNLEEENRHLLDQIQTLMLQNRTLLEQNMESKDLFHVEQRQYIDKLNELRRQKEKLEEKIMDQYKFYDPSPPRRRGNWITLKMRKLIKSKKDINRERQKSLTLTPTRSDSSEGFLQLPHQDSQDSSSVGSNSLEDGQTLGTKKSSMVALKRLPFLRNRPKDKDKMKACYRRSMSMNDLVQSMVLAGQWTGSTENLEVPDDISTGKRRKELGAMAFSTTAINFSTVNSSAGFRSKQLVNNKDTTSFEDISPQGVSDDSSTGSRVHASRPASLDSGRTSTSNSNNNASLHEVKAGAVNNQSRPQSHSSGEFSLLHDHEAWSSSGSSPIQYLKRQTRSSPVLQHKISETLESRHHKIKTGSPGSEVVTLQQFLEESNKLTSVQIKSSSQENLLDEVMKSLSVSSDFLGKDKPVSCGLARSVSGKTPGDFYDRRTTKPEFLRPGPRKTEDTYFISSAGKPTPGTQGKIKLVKESSLSRQSKDSNPYATLPRASSVISTAEGTTRRTSIHDFLTKDSRLPISVDSPPAAADSNTTAASNVDKVQESRNSKSRSREQQSS",
# CERKL_HUMAN
"MPWRRRRNRVSALEGGREEEAPPEAAAVPPALLTSPQQTEAAAERILLRGIFEIGRDSCDVVLSERALRWRPIQPERPAGDSKYDLLCKEEFIELKDIFSVKLKRRCSVKQQRSGTLLGITLFICLKKEQNKLKNSTLDLINLSEDHCDIWFRQFKKILAGFPNRPKSLKILLNPQSHKKEATQVYYEKVEPLLKLAGIKTDVTIMEYEGHALSLLKECELQGFDGGHRKPLFAIHWSVQRLFTGMQTLEPSVVCVGGDGSASEVAHALLLRAQKNAGMETDRILTPVRAQLPLGLIPAGSTNVLAHSLHGVPHVITATLHIIMGHVQLVDVCTFSTAGKLLRFGFSAMFGFGGRTLALAEKYRWMSPNQRRDFAVVKALAKLKAEDCEISFLPFNSSDDVQERRAQGSPKSDCNDQWQMIQGQFLNVSIMAIPCLCSVAPRGLAPNTRLNNGSMALIIARNTSRPEFIKHLKRYASVKNQFNFPFVETYTVEEVKVHPRNNTGGYNPEEEEDETASENCFPWNVDGDLMEVASEVHIRLHPRLISLYGGSMEEMIPK",
# TPC3L_HUMAN
"MSRPAHRRPEYHKINKDLFVLTYGALVAQLCKDYEKDEDVNQYLDKMGYGIGTRLVEDFLARSCVGRCHSYSEIIDIIAQVAFKMYLGITPSVTCNNSSKNEFSLILEKNPLVEFVEELPAGRSSLCYCNLLCGIIRGALEMVHLAADVTFLQDRLKGDSVTEIGITFLKKRDEKKYRGKK",
# MCTP2_HUMAN
"MDLDKPSVWGSLKQRTRPLLINLSKKKVKKNPSKPPDLRARHHLDRRLSLSVPDLLEAEALAPEGRPYSGPQSSYTSVPSSLSTAGIFPKSSSSSLKQSEEELDWSQEEASHLHVVETDSEEAYASPAERRRVSSNGIFDLQKTSLGGDAPEEPEKLCGSSDLNASMTSQHFEEQSVPGEASDGLSNLPSPFAYLLTIHLKEGRNLVVRDRCGTSDPYVKFKLNGKTLYKSKVIYKNLNPVWDEIVVLPIQSLDQKLRVKVYDRDLTTSDFMGSAFVILSDLELNRTTEHILKLEDPNSLEDDMGVIVLNLNLVVKQGDFKRHRWSNRKRLSASKSSLIRNLRLSESLKKNQLWNGIISITLLEGKNVSGGSMTEMFVQLKLGDQRYKSKTLCKSANPQWQEQFDFHYFSDRMGILDIEVWGKDNKKHEERLGTCKVDISALPLKQANCLELPLDSCLGALLMLVTLTPCAGVSVSDLCVCPLADLSERKQITQRYCLQNSLKDVKDVGILQVKVLKAADLLAADFSGKSDPFCLLELGNDRLQTHTVYKNLNPEWNKVFTFPIKDIHDVLEVTVFDEDGDKPPDFLGKVAIPLLSIRDGQPNCYVLKNKDLEQAFKGVIYLEMDLIYNPVKASIRTFTPREKRFVEDSRKLSKKILSRDVDRVKRITMAIWNTMQFLKSCFQWESTLRSTIAFAVFLITVWNFELYMIPLALLLIFVYNFIRPVKGKVSSIQDSQESTDIDDEEDEDDKESEKKGLIERIYMVQDIVSTVQNVLEEIASFGERIKNTFNWTVPFLSSLACLILAAATIILYFIPLRYIILIWGINKFTKKLRNPYSIDNNELLDFLSRVPSDVQKVQYAELKLCSSHSPLRKKRSAL",
# ANO7_HUMAN
"MRMAATAWAGLQGPPLPTLCPAVRTGLYCRDQAHAERWAMTSETSSGSHCARSRMLRRRAQEEDSTVLIDVSPPEAEKRGSYGSTAHASEPGGQQAAACRAGSPAKPRIADFVLVWEEDLKLDRQQDSAARDRTDMHRTWRETFLDNLRAAGLCVDQQDVQDGNTTVHYALLSASWAVLCYYAEDLRLKLPLQELPNQASNWSAGLLAWLGIPNVLLEVVPDVPPEYYSCRFRVNKLPRFLGSDNQDTFFTSTKRHQILFEILAKTPYGHEKKNLLGIHQLLAEGVLSAAFPLHDGPFKTPPEGPQAPRLNQRQVLFQHWARWGKWNKYQPLDHVRRYFGEKVALYFAWLGFYTGWLLPAAVVGTLVFLVGCFLVFSDIPTQELCGSKDSFEMCPLCLDCPFWLLSSACALAQAGRLFDHGGTVFFSLFMALWAVLLLEYWKRKSATLAYRWDCSDYEDTEERPRPQFAASAPMTAPNPITGEDEPYFPERSRARRMLAGSVVIVVMVAVVVMCLVSIILYRAIMAIVVSRSGNTLLAAWASRIASLTGSVVNLVFILILSKIYVSLAHVLTRWEMHRTQTKFEDAFTLKVFIFQFVNFYSSPVYIAFFKGRFVGYPGNYHTLFGVRNEECAAGGCLIELAQELLVIMVGKQVINNMQEVLIPKLKGWWQKFRLRSKKRKAGASAGASQGPWEDDYELVPCEGLFDEYLEMVLQFGFVTIFVAACPLAPLFALLNNWVEIRLDARKFVCEYRRPVAERAQDIGIWFHILAGLTHLAVISNAFLLAFSSDFLPRAYYRWTRAHDLRGFLNFTLARAPSSFAAAHNRTCRYRAFRDDDGHYSQTYWNLLAIRLAFVIVFEHVVFSVGRLLDLLVPDIPESVEIKVKREYYLAKQALAENEVLFGTNGTKDEQPEGSELSSHWTPFTVPKASQLQQ",
# CAVN1_HUMAN
"MEDPTLYIVERPLPGYPDAEAPEPSSAGAQAAEEPSGAGSEELIKSDQVNGVLVLSLLDKIIGAVDQIQLTQAQLEERQAEMEGAVQSIQGELSKLGKAHATTSNTVSKLLEKVRKVSVNVKTVRGSLERQAGQIKKLEVNEAELLRRRNFKVMIYQDEVKLPAKLSISKSLKESEALPEKEGEELGEGERPEEDAAALELSSDEAVEVEEVIEESRAERIKRSGLRRVDDFKKAFSKEKMEKTKVRTRENLEKTRLKTKENLEKTRHTLEKRMNKLGTRLVPAERREKLKTSRDKLRKSFTPDHVVYARSKTAVYKVPPFTFHVKKIREGQVEVLKATEMVEVGADDDEGGAERGEAGDLRRGSSPDVHALLEITEESDAVLVDKSDSD",
# KCIP4_HUMAN
"MNVRRVESISAQLEEASSTGGFLYAQNSTKRSIKERLMKLLPCSAAKTSSPAIQNSVEDELEMATVRHRPEALELLEAQSKFTKKELQILYRGFKNECPSGVVNEETFKEIYSQFFPQGDSTTYAHFLFNAFDTDHNGAVSFEDFIKGLSILLRGTVQEKLNWAFNLYDINKDGYITKEEMLDIMKAIYDMMGKCTYPVLKEDAPRQHVETFFQKMDKNKDGVVTIDEFIESCQKDENIMRSMQLFENVI",
# CASPC_HUMAN
"MADEKPSNGVLVHMVKLLIKTFLDGIFDDLMENNVLNTDEIHLIGKCLKFVVSNAENLVDDITETAQIAGKIFREHLWNSKKQLSSDISSDGEREANMPGLNIRNKEFNYLHNRNGSELDLLGMRDLLENLGYSVVIKENLTAQEMETALRQFAAHPEHQSSDSTFLVFMSHSILNGICGTKHWDQEPDVLHDDTIFEIFNNRNCQSLKDKPKVIIMQACRGNGAGIVWFTTDSGKASADTHGRLLQGNICNDAVTKAHVEKDFIAFKSSTPHNVSWRHETNGSVFISQIIYYFREYSWSHHLEEIFQKVQHSFETPNILTQLPTIERLSMTRYFYLFPGN",
# NHLC1_HUMAN
"MAAEASESGPALHELMREAEISLLECKVCFEKFGHRQQRRPRNLSCGHVVCLACVAALAHPRTLALECPFCRRACRGCDTSDCLPVLHLIELLGSALRQSPAAHRAAPSAPGALTCHHTFGGWGTLVNPTGLALCPKTGRVVVVHDGRRRVKIFDSGGGCAHQFGEKGDAAQDIRYPVDVTITNDCHVVVTDAGDRSIKVFDFFGQIKLVIGGQFSLPWGVETTPQNGIVVTDAEAGSLHLLDVDFAEGVLRRTERLQAHLCNPRGVAVSWLTGAIAVLEHPLALGTGVCSTRVKVFSSSMQLVGQVDTFGLSLYFPSKITASAVTFDHQGNVIVADTSGPAILCLGKPEEFPVPKPMVTHGLSHPVALTFTKENSLLVLDTASHSIKVYKVDWG",
# GGYF2_HUMAN
"MAAETQTLNFGPEWLRALSSGGSITSPPLSPALPKYKLADYRYGREEMLALFLKDNKIPSDLLDKEFLPILQEEPLPPLALVPFTEEEQRNFSMSVNSAAVLRLTGRGGGGTVVGAPRGRSSSRGRGRGRGECGFYQRSFDEVEGVFGRGGGREMHRSQSWEERGDRRFEKPGRKDVGRPNFEEGGPTSVGRKHEFIRSESENWRIFREEQNGEDEDGGWRLAGSRRDGERWRPHSPDGPRSAGWREHMERRRRFEFDFRDRDDERGYRRVRSGSGSIDDDRDSLPEWCLEDAEEEMGTFDSSGAFLSLKKVQKEPIPEEQEMDFRPVDEGEECSDSEGSHNEEAKEPDKTNKKEGEKTDRVGVEASEETPQTSSSSARPGTPSDHQSQEASQFERKDEPKTEQTEKAEEETRMENSLPAKVPSRGDEMVADVQQPLSQIPSDTASPLLILPPPVPNPSPTLRPVETPVVGAPGMGSVSTEPDDEEGLKHLEQQAEKMVAYLQDSALDDERLASKLQEHRAKGVSIPLMHEAMQKWYYKDPQGEIQGPFNNQEMAEWFQAGYFTMSLLVKRACDESFQPLGDIMKMWGRVPFSPGPAPPPHMGELDQERLTRQQELTALYQMQHLQYQQFLIQQQYAQVLAQQQKAALSSQQQQQLALLLQQFQTLKMRISDQNIIPSVTRSVSVPDTGSIWELQPTASQPTVWEGGSVWDLPLDTTTPGPALEQLQQLEKAKAAKLEQERREAEMRAKREEEERKRQEELRRQQEEILRRQQEEERKRREEEELARRKQEEALRRQREQEIALRRQREEEERQQQEEALRRLEERRREEEERRKQEELLRKQEEEAAKWAREEEEAQRRLEENRLRMEEEAARLRHEEEERKRKELEVQRQKELMRQRQQQQEALRRLQQQQQQQQLAQMKLPSSSTWGQQSNTTACQSQATLSLAEIQKLEEERERQLREEQRRQQRELMKALQQQQQQQQQKLSGWGNVSKPSGTTKSLLEIQQEEARQMQKQQQQQQQHQQPNRARNNTHSNLHTSIGNSVWGSINTGPPNQWASDLVSSIWSNADTKNSNMGFWDDAVKEVGPRNSTNKNKNNASLSKSVGVSNRQNKKVEEEEKLLKLFQGVNKAQDGFTQWCEQMLHALNTANNLDVPTFVSFLKEVESPYEVHDYIRAYLGDTSEAKEFAKQFLERRAKQKANQQRQQQQLPQQQQQQPPQQPPQQPQQQDSVWGMNHSTLHSVFQTNQSNNQQSNFEAVQSGKKKKKQKMVRADPSLLGFSVNASSERLNMGEIETLDDY",
# NBEL2_HUMAN
"MAASERLYELWLLYYAQKDLGYLQQWLKAFVGAFKKSISLSSLEPRRPEEAGAEVPLLPLDELHVLAEQLHQADLEQALLLLKLFIILCRNLENIEAGRGQVLVPRVLALLTKLVAELKGCPPPQGRGTQLENVALHALLLCEGLFDPYQTWRRQRSGEVISSKEKSKYKFPPAALPQEFSAFFQESLQNADHLPPILLLRLIHLFCAVLAGGKENGQMAVSDGSVKGLLSVVRGWSRGPAPDPCLVPLALEALVGAVHVLHASRAPPRGPELRALLESYFHVLNADWPAGLSSGPEEALVTLRVSMLDAIPMMLACEDRPVLQATFLSNNCFEHLTRLIQNSKLYLQSRAPPEGDSDLATRLLTEPDVQKVLDQDTDAIAVHVVRVLTCIMSDSPSAKEVFKERIGYPHLQEVLQSHGPPTHRLLQELLNMAVEGDHSMCPPPPIRNEQPVLVLAQWLPSLPTAELRLFLAQRLRWLCDSCPASRATCVQAGLVGCLLETLSTGLALEARCQEQLLALLQALGRVSIRPMELRHLLRPRPGLDSEPGGAEAGKARHAGAVIRTLSGMARHQGPARALRYFDLTPSMAGIMVPPVQRWPGPGFTFHAWLCLHPMDTAPTPAPTRPLQRKQLYSFFTSSGSGFEAFFTAAGTLVVAVCTRKEYLTMSLPEVSFADSAWHCVAIVHVPGRRPFSQNLVHVYKDGHLVKTAPLRCPSLSEPFSSCCIGSAGYRTTTTTTGLPTPPVPATLAYTHPALTRSQSVPASTGLGWGSGLVAPLQEGSIDSTLAGTQDTRWGSPTSLEGELGAVAIFHEALQATALRTLCTLGPNETAPFKPEGELHELSTRLLLHYSPQACKNNICLDLSPSHGLDGRLTGHRVETWDVKDVVNCVGGMGALLPLLERVAAQPKEAEAGPAETHDLVGPELTSGHNTQGLVLPLGKSSEERMERNAVAAFLLMLRNFLQGHMVNQESLVQCQGPAIIGALLRKVPSWAMDMNVLMSAQLLMEQVAAEGSGPLLYLLYQHLLFNFHLWTLSDFAVRLGHIQYMSSIVREHRQKLRKKYGVQFILDALRTHYSPQRERPLAADDLRTVQTSLLGLAREFLVRSLSADDVQVTQTMLSFLAATGDDGQAVGALDLLLALLHGSLVQESLAVFLLEPGNLEVLLALLVRPGSLPLLPDRVCKILRRLQQNERLPERSRQRLRLRECGLQGLVACLPEGTVSPQLCQGLYKLFLGADCLNLSDLLAVVQLSLQADLSVRLDICRQLFHLIYGQPDVVRLLARQAGWQDVLTRLYVLEAATAGSPPPSSPESPTSPKPAPPKPPTESPAEPSDVFLPSEAPCPDPDGFYHALSPFCTPFDLGLERSSVGSGNTAGGGGSSGTLTPASQPGTPSPLDGPRPFPAAPGRHSSSLSNVLEDGSLPEPTISGDDTSNTSNPQQTSEEELCNLLTNVLFSVTWRGVEGSDEAAWRERGQVFSVLTQLGASATLVRPPDCIKRSLLEMMLESALTDIKEAPVGVLASLTQQALWLLRLLQDFLCAEGHGNQELWSEKLFEGVCSLLDRLGAWPHLANGTADLREMAQIGLRLVLGYILLEDPQLHAQAYVRLHMLLQTAVPARREEACYVLSKLEAALGRVLNTSSLESATDEAGSPLAAAAAAAAAERCSWLVPLVRTLLDRAYEPLGLQWGLPSLPPTNGSPTFFEDFQAFCATPEWRHFIDKQVQPTMSQFEMDTYAKSHDLMSGFWNACYDMLMSSGQRRQWERAQSRRAFQELVLEPAQRRARLEGLRYTAVLKQQATQHSMALLHWGALWRQLASPCGAWALRDTPIPRWKLSSAETYSRMRLKLVPNHHFDPHLEASALRDNLGEVPLTPTEEASLPLAVTKEAKVSTPPELLQEDQLGEDELAELETPMEAAELDEQREKLVLSAECQLVTVVAVVPGLLEVTTQNVYFYDGSTERVETEEGIGYDFRRPLAQLREVHLRRFNLRRSALELFFIDQANYFLNFPCKVGTTPVSSPSQTPRPQPGPIPPHTQVRNQVYSWLLRLRPPSQGYLSSRSPQEMLRASGLTQKWVQREISNFEYLMQLNTIAGRTYNDLSQYPVFPWVLQDYVSPTLDLSNPAVFRDLSKPIGVVNPKHAQLVREKYESFEDPAGTIDKFHYGTHYSNAAGVMHYLIRVEPFTSLHVQLQSGRFDCSDRQFHSVAAAWQARLESPADVKELIPEFFYFPDFLENQNGFDLGCLQLTNEKVGDVVLPPWASSPEDFIQQHRQALESEYVSAHLHEWIDLIFGYKQRGPAAEEALNVFYYCTYEGAVDLDHVTDERERKALEGIISNFGQTPCQLLKEPHPTRLSAEEAAHRLARLDTNSPSIFQHLDELKAFFAEVVSDGVPLVLALVPHRQPHSFITQGSPDLLVTVSASGLLGTHSWLPYDRNISNYFSFSKDPTMGSHKTQRLLSGPWVPGSGVSGQALAVAPDGKLLFSGGHWDGSLRVTALPRGKLLSQLSCHLDVVTCLALDTCGIYLISGSRDTTCMVWRLLHQGGLSVGLAPKPVQVLYGHGAAVSCVAISTELDMAVSGSEDGTVIIHTVRRGQFVAALRPLGATFPGPIFHLALGSEGQIVVQSSAWERPGAQVTYSLHLYSVNGKLRASLPLAEQPTALTVTEDFVLLGTAQCALHILQLNTLLPAAPPLPMKVAIRSVAVTKERSHVLVGLEDGKLIVVVAGQPSEVRSSQFARKLWRSSRRISQVSSGETEYNPTEAR",
# FGD5_HUMAN
"MFRGPKPPIAPKPRLTAPNEWRASVYLNDSLNKCSNGRLPCVDRGLDEGPRSIPKCSESETDEDYIVVPRVPLREDEPKDEGSVGNKALVSPESSAEEEEEREEGGEACGLEGTGAGEDSVAPAAPGAGALSREGEEGTDLALEDEGEGCADEPGTLEQVSRSEEEEKLVQPHRECSLEDSGPWAGEGVFQSDLLLPHIHGEDQEPPDTPGEAEEDDEEGCASTDPAGADEGSGPDRPTEDMGQDAEDTSEEPPEKEELAGVQEAETATDCPEVLEEGCEEATGVTGGEQVDLSEPPDHEKKTNQEVAAATLEDHAQDESAEESCQIVPFENDCMEDFVTSLTGSPYEFFPTESTSFCSESCSPLSESAKGLESEQAPKLGLRAEENPMVGALCGQCGSLQGGAAEGPAAPDVVVVLEEEALDDALANPYVMGVGLPGQAAPGEGGQAASDALGGYGSKEELNCEAEGGLVPADRKNTSTRVRPHSGKVAGYVPETVPEETGPEAGSSAPGIGGAAEEVGKTLLSLEGKPLEASRALPAKPRAFTLYPRSFSVEGREIPVSVYQEPEGSGLDDHRIKRKEDNLSLSCVIGSSGSFSQRNHLPSSGTSTPSSMVDIPPPFDLACITKKPITKSSPSLLIESDSPDKYKKKKSSFKRFLALTFKKKTENKLHVDVNVSSSRSSSESSYHGPSRILEVDRRSLSNSPQLKSRTGKLRASESPSSLIFYRDGKRKGVPFSRTVSRVESFEDRSRPPFLPLPLTKPRSISFPSADTSDYENIPAMNSDYENIQIPPRRPARAGAFTKLFEDQSRALSTANENDGYVDMSSFNAFESKQQSADQDAESAYTEPYKVCPISSAAPKEDLTSDEEQRSSEEEDSASRDPSVTHKVEGQSRALVIAQELLSSEKAYVEMLQHLNLDFHGAVMRALDDMDHEGRDTLAREELRQGLSELPAIHDLHQGILEELEERLSNWESQQKVADVFLAREQGFDHHATHILQFDRYLGLLSENCLHSPRLAAAVREFEQSVQGGSQTAKHRLLRVVQRLFQYQVLLTDYLNNLCPDSAEYDNTQGALSLISKVTDRANDSMEQGENLQKLVHIEHSVRGQGDLLQPGREFLKEGTLMKVTGKNRRPRHLFLMNDVLLYTYPQKDGKYRLKNTLAVANMKVSRPVMEKVPYALKIETSESCLMLSASSCAERDEWYGCLSRALPEDYKAQALAAFHHSVEIRERLGVSLGERPPTLVPVTHVMMCMNCGCDFSLTLRRHHCHACGKIVCRNCSRNKYPLKYLKDRMAKVCDGCFGELKKRGRAVPGLMRERPVSMSFPLSSPRFSGSAFSSVFQSINPSTFKKQKKVPSALTEVAASGEGSAISGYLSRCKRGKRHWKKLWFVIKGKVLYTYMASEDKVALESMPLLGFTIAPEKEEGSSEVGPIFHLYHKKTLFYSFKAEDTNSAQRWIEAMEDASVL",
# UN13D_HUMAN
"MATLLSHPQQRPPFLRQAIKIRRRRVRDLQDPPPQMAPEIQPPSHHFSPEQRALLYEDALYTVLHRLGHPEPNHVTEASELLRYLQEAFHVEPEEHQQTLQRVRELEKPIFCLKATVKQAKGILGKDVSGFSDPYCLLGIEQGVGVPGGSPGSRHRQKAVVRHTIPEEETHRTQVITQTLNPVWDETFILEFEDITNASFHLDMWDLDTVESVRQKLGELTDLHGLRRIFKEARKDKGQDDFLGNVVLRLQDLRCREDQWYPLEPRTETYPDRGQCHLQFQLIHKRRATSASRSQPSYTVHLHLLQQLVSHEVTQHEAGSTSWDGSLSPQAATVLFLHATQKDLSDFHQSMAQWLAYSRLYQSLEFPSSCLLHPITSIEYQWIQGRLKAEQQEELAASFSSLLTYGLSLIRRFRSVFPLSVSDSPARLQSLLRVLVQMCKMKAFGELCPNTAPLPQLVTEALQTGTTEWFHLKQQHHQPMVQGIPEAGKALLGLVQDVIGDLHQCQRTWDKIFHNTLKIHLFSMAFRELQWLVAKRVQDHTTVVGDVVSPEMGESLFQLYISLKELCQLRMSSSERDGVLALDNFHRWFQPAIPSWLQKTYNEALARVQRAVQMDELVPLGELTKHSTSAVDLSTCFAQISHTARQLDWPDPEEAFMITVKFVEDTCRLALVYCSLIKARARELSSGQKDQGQAANMLCVVVNDMEQLRLVIGKLPAQLAWEALEQRVGAVLEQGQLQNTLHAQLQSALAGLGHEIRTGVRTLAEQLEVGIAKHIQKLVGVRESVLPEDAILPLMKFLEVELCYMNTNLVQENFSSLLTLLWTHTLTVLVEAAASQRSSSLASNRLKIALQNLEICFHAEGCGLPPKALHTATFQALQRDLELQAASSRELIRKYFCSRIQQQAETTSEELGAVTVKASYRASEQKLRVELLSASSLLPLDSNGSSDPFVQLTLEPRHEFPELAARETQKHKKDLHPLFDETFEFLVPAEPCRKAGACLLLTVLDYDTLGADDLEGEAFLPLREVPGLSGSEEPGEVPQTRLPLTYPAPNGDPILQLLEGRKGDREAQVFVRLRRHRAKQASQHALRPAP",
# U17L1_HUMAN
"MGDDSLYLGGEWQFNHFSKLTSSRPDAAFAEIQRTSLPEKSPLSSETRVDLCDDLAPVARQLAPREKLPLSSRRPAAVGAGLQNMGNTCYENASLQCLTYTLPLANYMLSREHSQTCQRPKCCMLCTMQAHITWALHSPGHVIQPSQALAAGFHRGKQEDVHEFLMFTVDAMKKACLPGHKQVDHHCKDTTLIHQIFGGCWRSQIKCLHCHGISDTFDPYLDIALDIQAAQSVKQALEQLVKPEELNGENAYHCGLCLQRAPASNTLTLHTSAKVLILVLKRFSDVAGNKLAKNVQYPECLDMQPYMSQQNTGPLVYVLYAVLVHAGWSCHDGHYFSYVKAQEVQWYKMDDAEVTVCSIISVLSQQAYVLFYIQKSEWERHSESVSRGREPRALGAEDTDRRAKQGELKRDHPCLQAPELDEHLVERATQESTLDHWKFLQEQNKTKPEFNVGKVEGTLPPNALVIHQSKYKCGMKNHHPEQQSSLLNLSSTTRTDQESMNTGTLASLQGRTRRAKGKNKHSKRALLVCQ",
# TPC6B_HUMAN
"MADEALFLLLHNEMVSGVYKSAEQGEVENGRCITKLENMGFRVGQGLIERFTKDTARFKDELDIMKFICKDFWTTVFKKQIDNLRTNHQGIYVLQDNKFRLLTQMSAGKQYLEHASKYLAFTCGLIRGGLSNLGIKSIVTAEVSSMPACKFQVMIQKL",
# TRXR3_HUMAN
"MERSPPQSPGPGKAGDAPNRRSGHVRGARVLSPPGRRARLSSPGPSRSSEAREELRRHLVGLIERSRVVIFSKSYCPHSTRVKELFSSLGVECNVLELDQVDDGARVQEVLSEITNQKTVPNIFVNKVHVGGCDQTFQAYQSGLLQKLLQEDLAYDYDLIIIGGGSGGLSCAKEAAILGKKVMVLDFVVPSPQGTSWGLGGTCVNVGCIPKKLMHQAALLGQALCDSRKFGWEYNQQVRHNWETMTKAIQNHISSLNWGYRLSLREKAVAYVNSYGEFVEHHKIKATNKKGQETYYTAAQFVIATGERPRYLGIQGDKEYCITSDDLFSLPYCPGKTLVVGASYVALECAGFLAGFGLDVTVMVRSILLRGFDQEMAEKVGSYMEQHGVKFLRKFIPVMVQQLEKGSPGKLKVLAKSTEGTETIEGVYNTVLLAIGRDSCTRKIGLEKIGVKINEKSGKIPVNDVEQTNVPYVYAVGDILEDKPELTPVAIQSGKLLAQRLFGASLEKCDYINVPTTVFTPLEYGCCGLSEEKAIEVYKKENLEIYHTLFWPLEWTVAGRENNTCYAKIICNKFDHDRVIGFHILGPNAGEVTQGFAAAMKCGLTKQLLDDTIGIHPTCGEVFTTLEITKSSGLDITQKGCUG",
# HPS6_HUMAN
"MKRSGTLRLLSDLSAFGGAARLRELVAGDSAVRVRGSPDGRHLLLLRPPGAVAPQLLVASRGPGAELERAWPAGQPSPLDAFFLPWPARPALVLVWESGLAEVWGAGVGPGWRPLQSTELCPGGGARVVAVAALRGRLVWCEERQARAEGPSGSPAAAFSHCVCVRTLEPSGEASTSLGRTHVLLHHCPAFGLLASCRQLFLVPTATTWPGVAHVLLIWSPGKGKVMVAAPRLGLSYSKSLNPGRGDTWDFRTLLRGLPGLLSPREPLAVHTWAPTPQGLLLLDFGGTVSLLQSHGGTRAVGTLQEAPVGPWGSAALGTFQGTLACVLGSTLELLDMGSGQLLERKVLSTDRVHLLEPPAPGMEDEEELETRGNLRLLSALGLFCVGWEAPQGVELPSAKDLVFEEACGYYQRRSLRGAQLTPEELRHSSTFRAPQALASILQGHLPPSALLTMLRTELRDYRGLEQLKAQLVAGDDEEAGWTELAEQEVARLLRTELIGDQLAQLNTVFQALPTAAWGATLRALQLQLDGNGKLRSQAPPDVWKKVLGGITAGKEPPNGILPPFELLCQCLCQLEPRWLPPFVELAQQQGGPGWGAGGPGLPLYRRALAVLGEEGTRPEALELELLLSSGRPKAVLQAVGQLVQKEQWDRALDAGLALGPSSPLLRSEIFKLLLAEFAQHRRLDAHLPLLCRLCPPELAPAELLLLLRTYLPDEVGPPTPFPEPGAEPPLTVGLLKALLEQTGAQGWLSGPVLSPYEDILWDPSTPPPTPPRDL",
# TPPC5_HUMAN
"MEARFTRGKSALLERALARPRTEVSLSAFALLFSELVQHCQSRVFSVAELQSRLAALGRQVGARVLDALVAREKGARRETKVLGALLFVKGAVWKALFGKEADKLEQANDDARTFYIIEREPLINTYISVPKENSTLNCASFTAGIVEAVLTHSGFPAKVTAHWHKGTTLMIKFEEAVIARDRALEGR",
# MSRB3_HUMAN
"MSPRRTLPRPLSLCLSLCLCLCLAAALGSAQSGSCRDKKNCKVVFSQQELRKRLTPLQYHVTQEKGTESAFEGEYTHHKDPGIYKCVVCGTPLFKSETKFDSGSGWPSFHDVINSEAITFTDDFSYGMHRVETSCSQCGAHLGHIFDDGPRPTGKRYCINSAALSFTPADSSGTAEGGSGVASPAQADKAEL",
# ABI1_HUMAN
"MAELQMLLEEEIPSGKRALIESYQNLTRVADYCENNYIQATDKRKALEETKAYTTQSLASVAYQINALANNVLQLLDIQASQLRRMESSINHISQTVDIHKEKVARREIGILTTNKNTSRTHKIIAPANMERPVRYIRKPIDYTVLDDVGHGVKWLKAKHGNNQPARTGTLSRTNPPTQKPPSPPMSGRGTLGRNTPYKTLEPVKPPTVPNDYMTSPARLGSQHSPGRTASLNQRPRTHSGSSGGSGSRENSGSSSIGIPIAVPTPSPPTIGPENISVPPPSGAPPAPPLAPLLPVSTVIAAPGSAPGSQYGTMTRQISRHNSTTSSTSSGGYRRTPSVTAQFSAQPHVNGGPLYSQNSISIAPPPPPMPQLTPQIPLTGFVARVQENIADSPTPPPPPPPDDIPMFDDSPPPPPPPPVDYEDEEAAVVQYNDPYADGDPAWAPKNYIEKVVAIYDYTKDKDDELSFMEGAIIYVIKKNDDGWYEGVCNRVTGLFPGNYVESIMHYTD",
# GIMA8_HUMAN
"MSEQSCQMSELRLLLLGKCRSGKSATGNAILGKHVFKSKFSDQTVIKMCQRESWVLRERKVVVIDTPDLFSSIACAEDKQRNIQHCLELSAPSLHALLLVIAIGHFTREDEETAKGIQQVFGAEARRHIIIVFTRKDDLGDDLLQDFIEKNKPLKQLVQDYEGRYCIFNNKTNSKDEQITQVLELLRKVESLVNTNGGPYHVNFKTEGSRFQDCVNEAASQEGDKPQGPRERQLQSTGPEQNPGTSELTVLLVGKRGAGKSAAGNSILGRQAFQTGFSEQSVTQSFLSESRSWRKKKVSIIDAPDISSLKNIDSEVRKHICTGPHAFLLVTPLGFYTKNDEAVLSTIQNNFGEKFFEYMIILLTRKEDLGDQDLDTFLRNSNKALYGLIQKCKNRYSAFNYRATGEEEQRQADELLEKIESMVHQNGNKHCVFREKETLNIVLVGRSGTGKSATGNSILGSLVFTSRLRAQPVTKTSQSGRRTWDGQEVVVVDTPSFNQMLDVEKDPSRLEEEVKRCLSCCEKGDTFFVLVFQLGRFTEEDKTAVAKLEAIFGADFTKYAIMLFTRKEDLGAGNLEDFMKNSDNKALRRIFKKCGRRVCAFNNKETGQAQETQVKALLTKVNDLRKESGWSGYPHTQENVSKLIKNVQEMSQAEKLLKNLIGILQ",
# PXL2B_HUMAN
"MSTVDLARVGACILKHAVTGEAVELRSLWREHACVVAGLRRFGCVVCRWIAQDLSSLAGLLDQHGVRLVGVGPEALGLQEFLDGDYFAGELYLDESKQLYKELGFKRYNSLSILPAALGKPVRDVAAKAKAVGIQGNLSGDLLQSGGLLVVSKGGDKVLLHFVQKSPGDYVPKEHILQVLGISAEVCASDPPQCDREV",
# RN128_HUMAN
"MGPPPGAGVSCRGGCGFSRLLAWCFLLALSPQAPGSRGAEAVWTAYLNVSWRVPHTGVNRTVWELSEEGVYGQDSPLEPVAGVLVPPDGPGALNACNPHTNFTVPTVWGSTVQVSWLALIQRGGGCTFADKIHLAYERGASGAVIFNFPGTRNEVIPMSHPGAVDIVAIMIGNLKGTKILQSIQRGIQVTMVIEVGKKHGPWVNHYSIFFVSVSFFIITAATVGYFIFYSARRLRNARAQSRKQRQLKADAKKAIGRLQLRTLKQGDKEIGPDGDSCAVCIELYKPNDLVRILTCNHIFHKTCVDPWLLEHRTCPMCKCDILKALGIEVDVEDGSVSLQVPVSNEISNSASSHEEDNRSETASSGYASVQGTDEPPLEEHVQSTNESLQLVNHEANSVAVDVIPHVDNPTFEEDETPNQETAVREIKS",
# PSMF1_HUMAN
"MAGLEVLFASAAPAITCRQDALVCFLHWEVVTHGYFGLGVGDQPGPNDKKSELLPAGWNNNKDLYVLRYEYKDGSRKLLVKAITVESSMILNVLEYGSQQVADLTLNLDDYIDAEHLGDFHRTYKNSEELRSRIVSGIITPIHEQWEKANVSSPHREFPPATAREVDPLRIPPHHPHTSRQPPWCDPLGPFVVGGEDLDPFGPRRGGMIVDPLRSGFPRALIDPSSGLPNRLPPGAVPPGARFDPFGPIGTSPPGPNPDHLPPPGYDDMYL",
# KIFA3_HUMAN
"MQGEDARYLKRKVKGGNIDVHPSEKALIVHYEVEATILGEMGDPMLGERKECQKIIRLKSLNANTDITSLARKVVEECKLIHPSKLNEVEQLLYYLQNRRDSLSGKEKKEKSSKPKDPPPFEGMEIDEVANINDMDEYIELLYEDIPDKVRGSALILQLARNPDNLEELLLNETALGALARVLREDWKQSVELATNIIYIFFCFSSFSQFHGLITHYKIGALCMNIIDHELKRHELWQEELSKKKKAVDEDPENQTLRKDYEKTFKKYQGLVVKQEQLLRVALYLLLNLAEDTRTELKMRNKNIVHMLVKALDRDNFELLILVVSFLKKLSIFMENKNDMVEMDIVEKLVKMIPCEHEDLLNITLRLLLNLSFDTGLRNKMVQVGLLPKLTALLGNDNYKQIAMCVLYHISMDDRFKSMFAYTDCIPQLMKMLFECSDERIDLELISFCINLAANKRNVQLICEGNGLKMLMKRALKFKDPLLMKMIRNISQHDGPTKNLFIDYVGDLAAQISNDEEEEFVIECLGTLANLTIPDLDWELVLKEYKLVPYLKDKLKPGAAEDDLVLEVVIMIGTVSMDDSCAALLAKSGIIPALIELLNAQQEDDEFVCQIIYVFYQMVFHQATRDVIIKETQAPAYLIDLMHDKNNEIRKVCDNTLDIIAEYDEEWAKKIQSEKFRWHNSQWLEMVESRQMDESEQYLYGDDRIEPYIHEGDILERPDLFYNSDGLIASEGAISPDFFNDYHLQNGDVVGQHSFPGSLGMDGFGQPVGILGRPATAYGFRPDEPYYYGYGS",
# CIDEC_HUMAN
"MEYAMKSLSLLYPKSLSRHVSVRTSVVTQQLLSEPSPKAPRARPCRVSTADRSVRKGIMAYSLEDLLLKVRDTLMLADKPFFLVLEEDGTTVETEEYFQALAGDTVFMVLQKGQKWQPPSEQGTRHPLSLSHKPAKKIDVARVTFDLYKLNPQDFIGCLNVKATFYDTYSLSYDLHCCGAKRIMKEAFRWALFSMQATGHVLLGTSCYLQQLLDATEEGQPPKGKASSLIPTCLKILQ",
# ACBG1_HUMAN
"MPRNSGAGYGCPHGDPSMLDSRETPQESRQDMIVRTTQEKLKTSSLTDRQPLSKESLNHALELSVPEKVNNAQWDAPEEALWTTRADGRVRLRIDPSCPQLPYTVHRMFYEALDKYGDLIALGFKRQDKWEHISYSQYYLLARRAAKGFLKLGLKQAHSVAILGFNSPEWFFSAVGTVFAGGIVTGIYTTSSPEACQYIAYDCCANVIMVDTQKQLEKILKIWKQLPHLKAVVIYKEPPPNKMANVYTMEEFMELGNEVPEEALDAIIDTQQPNQCCVLVYTSGTTGNPKGVMLSQDNITWTARYGSQAGDIRPAEVQQEVVVSYLPLSHIAAQIYDLWTGIQWGAQVCFAEPDALKGSLVNTLREVEPTSHMGVPRVWEKIMERIQEVAAQSGFIRRKMLLWAMSVTLEQNLTCPGSDLKPFTTRLADYLVLAKVRQALGFAKCQKNFYGAAPMMAETQHFFLGLNIRLYAGYGLSETSGPHFMSSPYNYRLYSSGKLVPGCRVKLVNQDAEGIGEICLWGRTIFMGYLNMEDKTCEAIDEEGWLHTGDAGRLDADGFLYITGRLKELIITAGGENVPPVPIEEAVKMELPIISNAMLIGDQRKFLSMLLTLKCTLDPDTSDQTDNLTEQAMEFCQRVGSRATTVSEIIEKKDEAVYQAIEEGIRRVNMNAAARPYHIQKWAILERDFSISGGELGPTMKLKRLTVLEKYKGIIDSFYQEQKM",
# SAHH3_HUMAN
"MSVQVVSAAAAAKVPEVELKDLSPSEAESQLGLSTAAVGAMAPPAGGGDPEAPAPAAERPPVPGPGSGPAAALSPAAGKVPQASAMKRSDPHHQHQRHRDGGEALVSPDGTVTEAPRTVKKQIQFADQKQEFNKRPTKIGRRSLSRSISQSSTDSYSSAASYTDSSDDETSPRDKQQKNSKGSSDFCVKNIKQAEFGRREIEIAEQEMPALMALRKRAQGEKPLAGAKIVGCTHITAQTAVLMETLGALGAQCRWAACNIYSTLNEVAAALAESGFPVFAWKGESEDDFWWCIDRCVNVEGWQPNMILDDGGDLTHWIYKKYPNMFKKIKGIVEESVTGVHRLYQLSKAGKLCVPAMNVNDSVTKQKFDNLYCCRESILDGLKRTTDMMFGGKQVVVCGYGEVGKGCCAALKAMGSIVYVTEIDPICALQACMDGFRLVKLNEVIRQVDIVITCTGNKNVVTREHLDRMKNSCIVCNMGHSNTEIDVASLRTPELTWERVRSQVDHVIWPDGKRIVLLAEGRLLNLSCSTVPTFVLSITATTQALALIELYNAPEGRYKQDVYLLPKKMDEYVASLHLPTFDAHLTELTDEQAKYLGLNKNGPFKPNYYRY",
# AIFM3_HUMAN
"MGGCFSKPKPVELKIEVVLPEKERGKEELSASGKGSPRAYQGNGTARHFHTEERLSTPHPYPSPQDCVEAAVCHVKDLENGQMREVELGWGKVLLVKDNGEFHALGHKCPHYGAPLVKGVLSRGRVRCPWHGACFNISTGDLEDFPGLDSLHKFQVKIEKEKVYVRASKQALQLQRRTKVMAKCISPSAGYSSSTNVLIVGAGAAGLVCAETLRQEGFSDRIVLCTLDRHLPYDRPKLSKSLDTQPEQLALRPKEFFRAYGIEVLTEAQVVTVDVRTKKVVFKDGFKLEYSKLLLAPGSSPKTLSCKGKEVENVFTIRTPEDANRVVRLARGRNVVVVGAGFLGMEVAAYLTEKAHSVSVVELEETPFRRFLGERVGRALMKMFENNRVKFYMQTEVSELRGQEGKLKEVVLKSSKVVRADVCVVGIGAVPATGFLRQSGIGLDSRGFIPVNKMMQTNVPGVFAAGDAVTFPLAWRNNRKVNIPHWQMAHAQGRVAAQNMLAQEAEMSTVPYLWTAMFGKSLRYAGYGEGFDDVIIQGDLEELKFVAFYTKGDEVIAVASMNYDPIVSKVAEVLASGRAIRKREVELFVLHSKTGDMSWLTGKGS",
# NLRP3_HUMAN
"MKMASTRCKLARYLEDLEDVDLKKFKMHLEDYPPQKGCIPLPRGQTEKADHVDLATLMIDFNGEEKAWAMAVWIFAAINRRDLYEKAKRDEPKWGSDNARVSNPTVICQEDSIEEEWMGLLEYLSRISICKMKKDYRKKYRKYVRSRFQCIEDRNARLGESVSLNKRYTRLRLIKEHRSQQEREQELLAIGKTKTCESPVSPIKMELLFDPDDEHSEPVHTVVFQGAAGIGKTILARKMMLDWASGTLYQDRFDYLFYIHCREVSLVTQRSLGDLIMSCCPDPNPPIHKIVRKPSRILFLMDGFDELQGAFDEHIGPLCTDWQKAERGDILLSSLIRKKLLPEASLLITTRPVALEKLQHLLDHPRHVEILGFSEAKRKEYFFKYFSDEAQARAAFSLIQENEVLFTMCFIPLVCWIVCTGLKQQMESGKSLAQTSKTTTAVYVFFLSSLLQPRGGSQEHGLCAHLWGLCSLAADGIWNQKILFEESDLRNHGLQKADVSAFLRMNLFQKEVDCEKFYSFIHMTFQEFFAAMYYLLEEEKEGRTNVPGSRLKLPSRDVTVLLENYGKFEKGYLIFVVRFLFGLVNQERTSYLEKKLSCKISQQIRLELLKWIEVKAKAKKLQIQPSQLELFYCLYEMQEEDFVQRAMDYFPKIEINLSTRMDHMVSSFCIENCHRVESLSLGFLHNMPKEEEEEEKEGRHLDMVQCVLPSSSHAACSHGLVNSHLTSSFCRGLFSVLSTSQSLTELDLSDNSLGDPGMRVLCETLQHPGCNIRRLWLGRCGLSHECCFDISLVLSSNQKLVELDLSDNALGDFGIRLLCVGLKHLLCNLKKLWLVSCCLTSACCQDLASVLSTSHSLTRLYVGENALGDSGVAILCEKAKNPQCNLQKLGLVNSGLTSVCCSALSSVLSTNQNLTHLYLRGNTLGDKGIKLLCEGLLHPDCKLQVLELDNCNLTSHCCWDLSTLLTSSQSLRKLSLGNNDLGDLGVMMFCEVLKQQSCLLQNLGLSEMYFNYETKSALETLQEEKPELTVVFEPSW",
# TPPC9_HUMAN
"MSVPDYMQCAEDHQTLLVVVQPVGIVSEENFFRIYKRICSVSQISVRDSQRVLYIRYRHHYPPENNEWGDFQTHRKVVGLITITDCFSAKDWPQTFEKFHVQKEIYGSTLYDSRLFVFGLQGEIVEQPRTDVAFYPNYEDCQTVEKRIEDFIESLFIVLESKRLDRATDKSGDKIPLLCVPFEKKDFVGLDTDSRHYKKRCQGRMRKHVGDLCLQAGMLQDSLVHYHMSVELLRSVNDFLWLGAALEGLCSASVIYHYPGGTGGKSGARRFQGSTLPAEAANRHRPGAQEVLIDPGALTTNGINPDTSTEIGRAKNCLSPEDIIDKYKEAISYYSKYKNAGVIELEACIKAVRVLAIQKRSMEASEFLQNAVYINLRQLSEEEKIQRYSILSELYELIGFHRKSAFFKRVAAMQCVAPSIAEPGWRACYKLLLETLPGYSLSLDPKDFSRGTHRGWAAVQMRLLHELVYASRRMGNPALSVRHLSFLLQTMLDFLSDQEKKDVAQSLENYTSKCPGTMEPIALPGGLTLPPVPFTKLPIVRHVKLLNLPASLRPHKMKSLLGQNVSTKSPFIYSPIIAHNRGEERNKKIDFQWVQGDVCEVQLMVYNPMPFELRVENMGLLTSGVEFESLPAALSLPAESGLYPVTLVGVPQTTGTITVNGYHTTVFGVFSDCLLDNLPGIKTSGSTVEVIPALPRLQISTSLPRSAHSLQPSSGDEISTNVSVQLYNGESQQLIIKLENIGMEPLEKLEVTSKVLTTKEKLYGDFLSWKLEETLAQFPLQPGKVATFTINIKVKLDFSCQENLLQDLSDDGISVSGFPLSSPFRQVVRPRVEGKPVNPPESNKAGDYSHVKTLEAVLNFKYSGGPGHTEGYYRNLSLGLHVEVEPSVFFTRVSTLPATSTRQCHLLLDVFNSTEHELTVSTRSSEALILHAGECQRMAIQVDKFNFESFPESPGEKGQFANPKQLEEERREARGLEIHSKLGICWRIPSLKRSGEASVEGLLNQLVLEHLQLAPLQWDVLVDGQPCDREAVAACQVGDPVRLEVRLTNRSPRSVGPFALTVVPFQDHQNGVHNYDLHDTVSFVGSSTFYLDAVQPSGQSACLGALLFLYTGDFFLHIRFHEDSTSKELPPSWFCLPSVHVCALEAQA",
# SRPK1_HUMAN
"MERKVLALQARKKRTKAKKDKAQRKSETQHRGSAPHSESDLPEQEEEILGSDDDEQEDPNDYCKGGYHLVKIGDLFNGRYHVIRKLGWGHFSTVWLSWDIQGKKFVAMKVVKSAEHYTETALDEIRLLKSVRNSDPNDPNREMVVQLLDDFKISGVNGTHICMVFEVLGHHLLKWIIKSNYQGLPLPCVKKIIQQVLQGLDYLHTKCRIIHTDIKPENILLSVNEQYIRRLAAEATEWQRSGAPPPSGSAVSTAPQPKPADKMSKNKKKKLKKKQKRQAELLEKRMQEIEEMEKESGPGQKRPNKQEESESPVERPLKENPPNKMTQEKLEESSTIGQDQTLMERDTEGGAAEINCNGVIEVINYTQNSNNETLRHKEDLHNANDCDVQNLNQESSFLSSQNGDSSTSQETDSCTPITSEVSDTMVCQSSSTVGQSFSEQHISQLQESIRAEIPCEDEQEQEHNGPLDNKGKSTAGNFLVNPLEPKNAEKLKVKIADLGNACWVHKHFTEDIQTRQYRSLEVLIGSGYNTPADIWSTACMAFELATGDYLFEPHSGEEYTRDEDHIALIIELLGKVPRKLIVAGKYSKEFFTKKGDLKHITKLKPWGLFEVLVEKYEWSQEEAAGFTDFLLPMLELIPEKRATAAECLRHPWLNS",
# PARK7_HUMAN
"MASKRALVILAKGAEEMETVIPVDVMRRAGIKVTVAGLAGKDPVQCSRDVVICPDASLEDAKKEGPYDVVVLPGGNLGAQNLSESAAVKEILKEQENRKGLIAAICAGPTALLAHEIGFGSKVTTHPLAKDKMMNGGHYTYSENRVEKDGLILTSRGPGTSFEFALAIVEALNGKEVAAQVKAPLVLKD",
# PLIN2_HUMAN
"MASVAVDPQPSVVTRVVNLPLVSSTYDLMSSAYLSTKDQYPYLKSVCEMAENGVKTITSVAMTSALPIIQKLEPQIAVANTYACKGLDRIEERLPILNQPSTQIVANAKGAVTGAKDAVTTTVTGAKDSVASTITGVMDKTKGAVTGSVEKTKSVVSGSINTVLGSRMMQLVSSGVENALTKSELLVEQYLPLTEEELEKEAKKVEGFDLVQKPSYYVRLGSLSTKLHSRAYQQALSRVKEAKQKSQQTISQLHSTVHLIEFARKNVYSANQKIQDAQDKLYLSWVEWKRSIGYDDTDESHCAEHIESRTLAIARNLTQQLQTTCHTLLSNIQGVPQNIQDQAKHMGVMAGDIYSVFRNAASFKEVSDSLLTSSKGQLQKMKESLDDVMDYLVNNTPLNWLVGPFYPQLTESQNAQDQGAEMDKSSQETQRSEHKTH",
# INP5K_HUMAN
"MSSRKLSGPKGRRLSIHVVTWNVASAAPPLDLSDLLQLNNRNLNLDIYVIGLQELNSGIISLLSDAAFNDSWSSFLMDVLSPLSFIKVSHVRMQGILLLVFAKYQHLPYIQILSTKSTPTGLFGYWGNKGGVNICLKLYGYYVSIINCHLPPHISNNYQRLEHFDRILEMQNCEGRDIPNILDHDLIIWFGDMNFRIEDFGLHFVRESIKNRCYGGLWEKDQLSIAKKHDPLLREFQEGRLLFPPTYKFDRNSNDYDTSEKKRKPAWTDRILWRLKRQPCAGPDTPIPPASHFSLSLRGYSSHMTYGISDHKPVSGTFDLELKPLVSAPLIVLMPEDLWTVENDMMVSYSSTSDFPSSPWDWIGLYKVGLRDVNDYVSYAWVGDSKVSCSDNLNQVYIDISNIPTTEDEFLLCYYSNSLRSVVGISRPFQIPPGSLREDPLGEAQPQI",
# S26A6_HUMAN
"MGLADASGPRDTQALLSATQAMDLRRRDYHMERPLLNQEHLEELGRWGSAPRTHQWRTWLQCSRARAYALLLQHLPVLVWLPRYPVRDWLLGDLLSGLSVAIMQLPQGLAYALLAGLPPVFGLYSSFYPVFIYFLFGTSRHISVGTFAVMSVMVGSVTESLAPQALNDSMINETARDAARVQVASTLSVLVGLFQVGLGLIHFGFVVTYLSEPLVRGYTTAAAVQVFVSQLKYVFGLHLSSHSGPLSLIYTVLEVCWKLPQSKVGTVVTAAVAGVVLVVVKLLNDKLQQQLPMPIPGELLTLIGATGISYGMGLKHRFEVDVVGNIPAGLVPPVAPNTQLFSKLVGSAFTIAVVGFAIAISLGKIFALRHGYRVDSNQELVALGLSNLIGGIFQCFPVSCSMSRSLVQESTGGNSQVAGAISSLFILLIIVKLGELFHDLPKAVLAAIIIVNLKGMLRQLSDMRSLWKANRADLLIWLVTFTATILLNLDLGLVVAVIFSLLLVVVRTQMPHYSVLGQVPDTDIYRDVAEYSEAKEVRGVKVFRSSATVYFANAEFYSDALKQRCGVDVDFLISQKKKLLKKQEQLKLKQLQKEEKLRKQAASPKGASVSINVNTSLEDMRSNNVEDCKMMQVSSGDKMEDATANGQEDSKAPDGSTLKALGLPQPDFHSLILDLGALSFVDTVCLKSLKNIFHDFREIEVEVYMAACHSPVVSQLEAGHFFDASITKKHLFASVHDAVTFALQHPRPVPDSPVSVTRL",
# SCAPE_HUMAN
"MMASFQRSNSHDKVRRIVAEEGRTARNLIAWSVPLESKDDDGKPKCQTGGKSKRTIQGTHKTTKQSTAVDCKITSSTTGDKHFDKSPTKTRHPRKIDLRARYWAFLFDNLRRAVDEIYVTCESDQSVVECKEVLMMLDNYVRDFKALIDWIQLQEKLEKTDAQSRPTSLAWEVKKMSPGRHVIPSPSTDRINVTSNARRSLNFGGSTGTVPAPRLAPTGVSWADKVKAHHTGSTASSEITPAQSCPPMTVQKASRKNERKDAEGWETVQRGRPIRSRSTAVMPKVSLATEATRSKDDSDKENVCLLPDESIQKGQFVGDGTSNTIESHPKDSLHSCDHPLAEKTQFTVSTLDDVKNSGSIRDNYVRTSEISAVHIDTECVSVMLQAGTPPLQVNEEKFPAEKARIENEMDPSDISNSMAEVLAKKEELADRLEKANEEAIASAIAEEEQLTREIEAEENNDINIETDNDSDFSASMGSGSVSFCGMSMDWNDVLADYEARESWRQNTSWGDIVEEEPARPPGHGIHMHEKLSSPSRKRTIAESKKKHEEKQMKAQQLREKLREEKTLKLQKLLEREKDVRKWKEELLDQRRRMMEEKLLHAEFKREVQLQAIVKKAQEEEAKVNEIAFINTLEAQNKRHDVLSKLKEYEQRLNELQEERQRRQEEKQARDEAVQERKRALEAERQARVEELLMKRKEQEARIEQQRQEKEKAREDAARERARDREERLAALTAAQQEAMEELQKKIQLKHDESIRRHMEQIEQRKEKAAELSSGRHANTDYAPKLTPYERKKQCSLCNVLISSEVYLFSHVKGRKHQQAVRENTSIQGRELSDEEVEHLSLKKYIIDIVVESTAPAEALKDGEERQKNKKKAKKIKARMNFRAKEYESLMETKNSGSDSPYKAKLQRLAKDLLKQVQVQDSGSWANNKVSALDRTLGEITRILEKENVADQIAFQAAGGLTALEHILQAVVPATNVNTVLRIPPKSLCNAINVYNLTCNNCSENCSDVLFSNKITFLMDLLIHQLTVYVPDENNTILGRNTNKQVFEGLTTGLLKVSAVVLGCLIANRPDGNCQPATPKIPTQEMKNKPSQGDPFNNRVQDLISYVVNMGLIDKLCACFLSVQGPVDENPKMAIFLQHAAGLLHAMCTLCFAVTGRSYSIFDNNRQDPTGLTAALQATDLAGVLHMLYCVLFHGTILDPSTASPKENYTQNTIQVAIQSLRFFNSFAALHLPAFQSIVGAEGLSLAFRHMASSLLGHCSQVSCESLLHEVIVCVGYFTVNHPDNQVIVQSGRHPTVLQKLCQLPFQYFSDPRLIKVLFPSLIAACYNNHQNKIILEQEMSCVLLATFIQDLAQTPGQAENQPYQPKGKCLGSQDYLELANRFPQQAWEEARQFFLKKEKK",
# ULBP1_HUMAN
"MAAAASPAFLLCLPLLHLLSGWSRAGWVDTHCLCYDFIITPKSRPEPQWCEVQGLVDERPFLHYDCVNHKAKAFASLGKKVNVTKTWEEQTETLRDVVDFLKGQLLDIQVENLIPIEPLTLQARMSCEHEAHGHGRGSWQFLFNGQKFLLFDSNNRKWTALHPGAKKMTEKWEKNRDVTMFFQKISLGDCKMWLEEFLMYWEQMLDPTKPPSLAPGTTQPKAMATTLSPWSLLIIFLCFILAGR",
# LSG1_HUMAN
"MGRRRAPAGGSLGRALMRHQTQRSRSHRHTDSWLHTSELNDGYDWGRLNLQSVTEQSSLDDFLATAELAGTEFVAEKLNIKFVPAEARTGLLSFEESQRIKKLHEENKQFLCIPRRPNWNQNTTPEELKQAEKDNFLEWRRQLVRLEEEQKLILTPFERNLDFWRQLWRVIERSDIVVQIVDARNPLLFRCEDLECYVKEMDANKENVILINKADLLTAEQRSAWAMYFEKEDVKVIFWSALAGAIPLNGDSEEEANRDDRQSNTTKFGHSSFDQAEISHSESEHLPARDSPSLSENPTTDEDDSEYEDCPEEEEDDWQTCSEEDGPKEEDCSQDWKESSTADSEARSRKTPQKRQIHNFSHLVSKQELLELFKELHTGRKVKDGQLTVGLVGYPNVGKSSTINTIMGNKKVSVSATPGHTKHFQTLYVEPGLCLCDCPGLVMPSFVSTKAEMTCSGILPIDQMRDHVPPVSLVCQNIPRHVLEATYGINIITPREDEDPHRPPTSEELLTAYGYMRGFMTAHGQPDQPRSARYILKDYVSGKLLYCHPPPGRDPVTFQHQHQRLLENKMNSDEIKMQLGRNKKAKQIENIVDKTFFHQENVRALTKGVQAVMGYKPGSGVVTASTASSENGAGKPWKKHGNRNKKEKSRRLYKHLDM",
# 5NT3A_HUMAN
"MRAPSMDRAAVARVGAVASASVCALVAGVVLAQYIFTLKRKTGRKTKIIEMMPEFQKSSVRIKNPTRVEEIICGLIKGGAAKLQIITDFDMTLSRFSYKGKRCPTCHNIIDNCKLVTDECRKKLLQLKEKYYAIEVDPVLTVEEKYPYMVEWYTKSHGLLVQQALPKAKLKEIVAESDVMLKEGYENFFDKLQQHSIPVFIFSAGIGDVLEEVIRQAGVYHPNVKVVSNFMDFDETGVLKGFKGELIHVFNKHDGALRNTEYFNQLKDNSNIILLGDSQGDLRMADGVANVEHILKIGYLNDRVDELLEKYMDSYDIVLVQDESLEVANSILQKIL",
# GBRL1_HUMAN
"MKFQYKEDHPFEYRKKEGEKIRKKYPDRVPVIVEKAPKARVPDLDKRKYLVPSDLTVGQFYFLIRKRIHLRPEDALFFFVNNTIPPTSATMGQLYEDNHEEDYFLYVAYSDESVYGK",
# TMT1A_HUMAN
"MELTIFILRLAIYILTFPLYLLNFLGLWSWICKKWFPYFLVRFTVIYNEQMASKKRELFSNLQEFAGPSGKLSLLEVGCGTGANFKFYPPGCRVTCIDPNPNFEKFLIKSIAENRHLQFERFVVAAGENMHQVADGSVDVVVCTLVLCSVKNQERILREVCRVLRPGGAFYFMEHVAAECSTWNYFWQQVLDPAWHLLFDGCNLTRESWKALERASFSKLKLQHIQAPLSWELVRPHIYGYAVK",
# DISC1_HUMAN
"MPGGGPQGAPAAAGGGGVSHRAGSRDCLPPAACFRRRRLARRPGYMRSSTGPGIGFLSPAVGTLFRFPGGVSGEESHHSESRARQCGLDSRGLLVRSPVSKSAAAPTVTSVRGTSAHFGIQLRGGTRLPDRLSWPCGPGSAGWQQEFAAMDSSETLDASWEAACSDGARRVRAAGSLPSAELSSNSCSPGCGPEVPPTPPGSHSAFTSSFSFIRLSLGSAGERGEAEGCPPSREAESHCQSPQEMGAKAASLDGPHEDPRCLSRPFSLLATRVSADLAQAARNSSRPERDMHSLPDMDPGSSSSLDPSLAGCGGDGSSGSGDAHSWDTLLRKWEPVLRDCLLRNRRQMEVISLRLKLQKLQEDAVENDDYDKAETLQQRLEDLEQEKISLHFQLPSRQPALSSFLGHLAAQVQAALRRGATQQASGDDTHTPLRMEPRLLEPTAQDSLHVSITRRDWLLQEKQQLQKEIEALQARMFVLEAKDQQLRREIEEQEQQLQWQGCDLTPLVGQLSLGQLQEVSKALQDTLASAGQIPFHAEPPETIRSLQERIKSLNLSLKEITTKVCMSEKFCSTLRKKVNDIETQLPALLEAKMHAISGNHFWTAKDLTEEIRSLTSEREGLEGLLSKLLVLSSRNVKKLGSVKEDYNRLRREVEHQETAYETSVKENTMKYMETLKNKLCSCKCPLLGKVWEADLEACRLLIQSLQLQEARGSLSVEDERQMDDLEGAAPPIPPRLHSEDKRKTPLKVLEEWKTHLIPSLHCAGGEQKEESYILSAELGEKCEDIGKKLLYLEDQLHTAIHSHDEDLIQSLRRELQMVKETLQAMILQLQPAKEAGEREAAASCMTAGVHEAQA",
# CS012_HUMAN
"MTIMVEDIMKLLCSLSGERKMKAAVKHSGKGALVTGAMAFVGGLVGGPPGLAVGGAVGGLLGAWMTSGQFKPVPQILMELPPAEQQRLFNEAAAIIRHLEWTDAVQLTALVMGSEALQQQLLAMLVNYVTKELRAEIQYDD",
# DUS2_HUMAN
"MILNSLSLCYHNKLILAPMVRVGTLPMRLLALDYGADIVYCEELIDLKMIQCKRVVNEVLSTVDFVAPDDRVVFRTCEREQNRVVFQMGTSDAERALAVARLVENDVAGIDVNMGCPKQYSTKGGMGAALLSDPDKIEKILSTLVKGTRRPVTCKIRILPSLEDTLSLVKRIERTGIAAIAVHGRKREERPQHPVSCEVIKAIADTLSIPVIANGGSHDHIQQYSDIEDFRQATAASSVMVARAAMWNPSIFLKEGLRPLEEVMQKYIRYAVQYDNHYTNTKYCLCQMLREQLESPQGRLLHAAQSSREICEAFGLGAFYEETTQELDAQQARLSAKTSEQTGEPAEDTSGVIKMAVKFDRRAYPAQITPKMCLLEWCRREKLAQPVYETVQRPLDRLFSSIVTVAEQKYQSTLWDKSKKLAEQAAAIVCLRSQGLPEGRLGEESPSLHKRKREAPDQDPGGPRAQELAQPGDLCKKPFVALGSGEESPLEGW",
# PK2L1_HUMAN
"MNAVGSPEGQELQKLGSGAWDNPAYSGPPSPHGTLRVCTISSTGPLQPQPKKPEDEPQETAYRTQVSSCCLHICQGIRGLWGTTLTENTAENRELYIKTTLRELLVYIVFLVDICLLTYGMTSSSAYYYTKVMSELFLHTPSDTGVSFQAISSMADFWDFAQGPLLDSLYWTKWYNNQSLGHGSHSFIYYENMLLGVPRLRQLKVRNDSCVVHEDFREDILSCYDVYSPDKEEQLPFGPFNGTAWTYHSQDELGGFSHWGRLTSYSGGGYYLDLPGSRQGSAEALRALQEGLWLDRGTRVVFIDFSVYNANINLFCVLRLVVEFPATGGAIPSWQIRTVKLIRYVSNWDFFIVGCEVIFCVFIFYYVVEEILELHIHRLRYLSSIWNILDLVVILLSIVAVGFHIFRTLEVNRLMGKLLQQPNTYADFEFLAFWQTQYNNMNAVNLFFAWIKIFKYISFNKTMTQLSSTLARCAKDILGFAVMFFIVFFAYAQLGYLLFGTQVENFSTFIKCIFTQFRIILGDFDYNAIDNANRILGPAYFVTYVFFVFFVLLNMFLAIINDTYSEVKEELAGQKDELQLSDLLKQGYNKTLLRLRLRKERVSDVQKVLQGGEQEIQFEDFTNTLRELGHAEHEITELTATFTKFDRDGNRILDEKEQEKMRQDLEEERVALNTEIEKLGRSIVSSPQGKSGPEAARAGGWVSGEEFYMLTRRVLQLETVLEGVVSQIDAVGSKLKMLERKGWLAPSPGVKEQAIWKHPQPAPAVTPDPWGVQGGQESEVPYKREEEALEERRLSRGEIPTLQRS",
# SYLC_HUMAN
"MAERKGTAKVDFLKKIEKEIQQKWDTERVFEVNASNLEKQTSKGKYFVTFPYPYMNGRLHLGHTFSLSKCEFAVGYQRLKGKCCLFPFGLHCTGMPIKACADKLKREIELYGCPPDFPDEEEEEEETSVKTEDIIIKDKAKGKKSKAAAKAGSSKYQWGIMKSLGLSDEEIVKFSEAEHWLDYFPPLAIQDLKRMGLKVDWRRSFITTDVNPYYDSFVRWQFLTLRERNKIKFGKRYTIYSPKDGQPCMDHDRQTGEGVGPQEYTLLKLKVLEPYPSKLSGLKGKNIFLVAATLRPETMFGQTNCWVRPDMKYIGFETVNGDIFICTQKAARNMSYQGFTKDNGVVPVVKELMGEEILGASLSAPLTSYKVIYVLPMLTIKEDKGTGVVTSVPSDSPDDIAALRDLKKKQALRAKYGIRDDMVLPFEPVPVIEIPGFGNLSAVTICDELKIQSQNDREKLAEAKEKIYLKGFYEGIMLVDGFKGQKVQDVKKTIQKKMIDAGDALIYMEPEKQVMSRSSDECVVALCDQWYLDYGEENWKKQTSQCLKNLETFCEETRRNFEATLGWLQEHACSRTYGLGTHLPWDEQWLIESLSDSTIYMAFYTVAHLLQGGNLHGQAESPLGIRPQQMTKEVWDYVFFKEAPFPKTQIAKEKLDQLKQEFEFWYPVDLRVSGKDLVPNHLSYYLYNHVAMWPEQSDKWPTAVRANGHLLLNSEKMSKSTGNFLTLTQAIDKFSADGMRLALADAGDTVEDANFVEAMADAGILRLYTWVEWVKEMVANWDSLRSGPASTFNDRVFASELNAGIIKTDQNYEKMMFKEALKTGFFEFQAAKDKYRELAVEGMHRELVFRFIEVQTLLLAPFCPHLCEHIWTLLGKPDSIMNASWPVAGPVNEVLIHSSQYLMEVTHDLRLRLKNYMMPAKGKKTDKQPLQKPSHCTIYVAKNYPPWQHTTLSVLRKHFEANNGKLPDNKVIASELGSMPELKKYMKKVMPFVAMIKENLEKMGPRILDLQLEFDEKAVLMENIVYLTNSLELEHIEVKFASEAEDKIREDCCPGKPLNVFRIEPGVSVSLVNPQPSNGHFSTKIEIRQGDNCDSIIRRLMKMNRGIKDLSKVKLMRFDDPLLGPRRVPVLGKEYTEKTPISEHAVFNVDLMSKKIHLTENGIRVDIGDTIIYLVH",
# SRP68_HUMAN
"MAAEKQVPGGGGGGGSGGGGGSGGGGSGGGRGAGGEENKENERPSAGSKANKEFGDSLSLEILQIIKESQQQHGLRHGDFQRYRGYCSRRQRRLRKTLNFKMGNRHKFTGKKVTEELLTDNRYLLLVLMDAERAWSYAMQLKQEANTEPRKRFHLLSRLRKAVKHAEELERLCESNRVDAKTKLEAQAYTAYLSGMLRFEHQEWKAAIEAFNKCKTIYEKLASAFTEEQAVLYNQRVEEISPNIRYCAYNIGDQSAINELMQMRLRSGGTEGLLAEKLEALITQTRAKQAATMSEVEWRGRTVPVKIDKVRIFLLGLADNEAAIVQAESEETKERLFESMLSECRDAIQVVREELKPDQKQRDYILEGEPGKVSNLQYLHSYLTYIKLSTAIKRNENMAKGLQRALLQQQPEDDSKRSPRPQDLIRLYDIILQNLVELLQLPGLEEDKAFQKEIGLKTLVFKAYRCFFIAQSYVLVKKWSEALVLYDRVLKYANEVNSDAGAFKNSLKDLPDVQELITQVRSEKCSLQAAAILDANDAHQTETSSSQVKDNKPLVERFETFCLDPSLVTKQANLVHFPPGFQPIPCKPLFFDLALNHVAFPPLEDKLEQKTKSGLTGYIKGIFGFRS",
# UBP25_HUMAN
"MTVEQNVLQQSAAQKHQQTFLNQLREITGINDTQILQQALKDSNGNLELAVAFLTAKNAKTPQQEETTYYQTALPGNDRYISVGSQADTNVIDLTGDDKDDLQRAIALSLAESNRAFRETGITDEEQAISRVLEASIAENKACLKRTPTEVWRDSRNPYDRKRQDKAPVGLKNVGNTCWFSAVIQSLFNLLEFRRLVLNYKPPSNAQDLPRNQKEHRNLPFMRELRYLFALLVGTKRKYVDPSRAVEILKDAFKSNDSQQQDVSEFTHKLLDWLEDAFQMKAEEETDEEKPKNPMVELFYGRFLAVGVLEGKKFENTEMFGQYPLQVNGFKDLHECLEAAMIEGEIESLHSENSGKSGQEHWFTELPPVLTFELSRFEFNQALGRPEKIHNKLEFPQVLYLDRYMHRNREITRIKREEIKRLKDYLTVLQQRLERYLSYGSGPKRFPLVDVLQYALEFASSKPVCTSPVDDIDASSPPSGSIPSQTLPSTTEQQGALSSELPSTSPSSVAAISSRSVIHKPFTQSRIPPDLPMHPAPRHITEEELSVLESCLHRWRTEIENDTRDLQESISRIHRTIELMYSDKSMIQVPYRLHAVLVHEGQANAGHYWAYIFDHRESRWMKYNDIAVTKSSWEELVRDSFGGYRNASAYCLMYINDKAQFLIQEEFNKETGQPLVGIETLPPDLRDFVEEDNQRFEKELEEWDAQLAQKALQEKLLASQKLRESETSVTTAQAAGDPEYLEQPSRSDFSKHLKEETIQIITKASHEHEDKSPETVLQSAIKLEYARLVKLAQEDTPPETDYRLHHVVVYFIQNQAPKKIIEKTLLEQFGDRNLSFDERCHNIMKVAQAKLEMIKPEEVNLEEYEEWHQDYRKFRETTMYLIIGLENFQRESYIDSLLFLICAYQNNKELLSKGLYRGHDEELISHYRRECLLKLNEQAAELFESGEDREVNNGLIIMNEFIVPFLPLLLVDEMEEKDILAVEDMRNRWCSYLGQEMEPHLQEKLTDFLPKLLDCSMEIKSFHEPPKLPSYSTHELCERFARIMLSLSRTPADGR",
# DDX41_HUMAN
"MEESEPERKRARTDEVPAGGSRSEAEDEDDEDYVPYVPLRQRRQLLLQKLLQRRRKGAAEEEQQDSGSEPRGDEDDIPLGPQSNVSLLDQHQHLKEKAEARKESAKEKQLKEEEKILESVAEGRALMSVKEMAKGITYDDPIKTSWTPPRYVLSMSEERHERVRKKYHILVEGDGIPPPIKSFKEMKFPAAILRGLKKKGIHHPTPIQIQGIPTILSGRDMIGIAFTGSGKTLVFTLPVIMFCLEQEKRLPFSKREGPYGLIICPSRELARQTHGILEYYCRLLQEDSSPLLRCALCIGGMSVKEQMETIRHGVHMMVATPGRLMDLLQKKMVSLDICRYLALDEADRMIDMGFEGDIRTIFSYFKGQRQTLLFSATMPKKIQNFAKSALVKPVTINVGRAGAASLDVIQEVEYVKEEAKMVYLLECLQKTPPPVLIFAEKKADVDAIHEYLLLKGVEAVAIHGGKDQEERTKAIEAFREGKKDVLVATDVASKGLDFPAIQHVINYDMPEEIENYVHRIGRTGRSGNTGIATTFINKACDESVLMDLKALLLEAKQKVPPVLQVLHCGDESMLDIGGERGCAFCGGLGHRITDCPKLEAMQTKQVSNIGRKDYLAHSSMDF",
# FBX2_HUMAN
"MDGDGDPESVGQPEEASPEEQPEEASAEEERPEDQQEEEAAAAAAYLDELPEPLLLRVLAALPAAELVQACRLVCLRWKELVDGAPLWLLKCQQEGLVPEGGVEEERDHWQQFYFLSKRRRNLLRNPCGEEDLEGWCDVEHGGDGWRVEELPGDSGVEFTHDESVKKYFASSFEWCRKAQVIDLQAEGYWEELLDTTQPAIVVKDWYSGRSDAGCLYELTVKLLSEHENVLAEFSSGQVAVPQDSDGGGWMEISHTFTDYGPGVRFVRFEHGGQDSVYWKGWFGARVTNSSVWVEP",
# TPC2L_HUMAN
"MAVCIAVIAKENYPLYIRSTPTENELKFHYMVHTSLDVVDEKISAMGKALVDQRELYLGLLYPTEDYKVYGYVTNSKVKFVMVVDSSNTALRDNEIRSMFRKLHNSYTDVMCNPFYNPGDRIQSSRAFDNMVTSMMIQVC",
# ASC_HUMAN
"MGRARDAILDALENLTAEELKKFKLKLLSVPLREGYGRIPRGALLSMDALDLTDKLVSFYLETYGAELTANVLRDMGLQEMAGQLQAATHQGSGAAPAGIQAPPQSAAKPGLHFIDQHRAALIARVTNVEWLLDALYGKVLTDEQYQAVRAEPTNPSKMRKLFSFTPAWNWTCKDLLLQALRESQSYLVEDLERS",
# UBQL1_HUMAN
"MAESGESGGPPGSQDSAAGAEGAGAPAAAASAEPKIMKVTVKTPKEKEEFAVPENSSVQQFKEEISKRFKSHTDQLVLIFAGKILKDQDTLSQHGIHDGLTVHLVIKTQNRPQDHSAQQTNTAGSNVTTSSTPNSNSTSGSATSNPFGLGGLGGLAGLSSLGLNTTNFSELQSQMQRQLLSNPEMMVQIMENPFVQSMLSNPDLMRQLIMANPQMQQLIQRNPEISHMLNNPDIMRQTLELARNPAMMQEMMRNQDRALSNLESIPGGYNALRRMYTDIQEPMLSAAQEQFGGNPFASLVSNTSSGEGSQPSRTENRDPLPNPWAPQTSQSSSASSGTASTVGGTTGSTASGTSGQSTTAPNLVPGVGASMFNTPGMQSLLQQITENPQLMQNMLSAPYMRSMMQSLSQNPDLAAQMMLNNPLFAGNPQLQEQMRQQLPTFLQQMQNPDTLSAMSNPRAMQALLQIQQGLQTLATEAPGLIPGFTPGLGALGSTGGSSGTNGSNATPSENTSPTAGTTEPGHQQFIQQMLQALAGVNPQLQNPEVRFQQQLEQLSAMGFLNREANLQALIATGGDINAAIERLLGSQPS",
# CHIP_HUMAN
"MKGKEEKEGGARLGAGGGSPEKSPSAQELKEQGNRLFVGRKYPEAAACYGRAITRNPLVAVYYTNRALCYLKMQQHEQALADCRRALELDGQSVKAHFFLGQCQLEMESYDEAIANLQRAYSLAKEQRLNFGDDIPSALRIAKKKRWNSIEERRIHQESELHSYLSRLIAAERERELEECQRNHEGDEDDSHVRAQQACIEAKHDKYMADMDELFSQVDEKRKKRDIPDYLCGKISFELMREPCITPSGITYDRKDIEEHLQRVGHFDPVTRSPLTQEQLIPNLAMKEVIDAFISENGWVEDY",
# ANGE1_HUMAN
"MIASCLCYLLLPATRLFRALSDAFFTCRKNVLLANSSSPQVEGDFAMAPRGPEQEECEGLLQQWREEGLSQVLSTASEGPLIDKGLAQSSLALLMDNPGEENAASEDRWSSRQLSDLRAAENLDEPFPEMLGEEPLLEVEGVEGSMWAAIPMQSEPQYADCAALPVGALATEQWEEDPAVLAWSIAPEPVPQEEASIWPFEGLGQLQPPAVEIPYHEILWREWEDFSTQPDAQGLKAGDGPQFQFTLMSYNILAQDLMQQSSELYLHCHPDILNWNYRFVNLMQEFQHWDPDILCLQEVQEDHYWEQLEPSLRMMGFTCFYKRRTGCKTDGCAVCYKPTRFRLLCASPVEYFRPGLELLNRDNVGLVLLLQPLVPEGLGQVSVAPLCVANTHILYNPRRGDVKLAQMAILLAEVDKVARLSDGSHCPIILCGDLNSVPDSPLYNFIRDGELQYHGMPAWKVSGQEDFSHQLYQRKLQAPLWPSSLGITDCCQYVTSCHPKRSERRKYGRDFLLRFRFCSIACQRPVGLVLMEGVTDTKPERPAGWAESVLEEDASELEPAFSRTVGTIQHCLHLTSVYTHFLPQRGRPEVTTMPLGLGMTVDYIFFSAESCENGNRTDHRLYRDGTLKLLGRLSLLSEEILWAANGLPNPFCSSDHLCLLASFGMEVTAP",
# POMP_HUMAN
"MNARGLGSELKDSIPVTELSASGPFESHDLLRKGFSCVKNELLPSHPLELSEKNFQLNQDKMNFSTLRNIQGLFAPLKLQMEFKAVQQVQRLPFLSSSNLSLDVLRGNDETIGFEDILNDPSQSEVMGEPHLMVEYKLGLL",
# TPPC4_HUMAN
"MAIFSVYVVNKAGGLIYQLDSYAPRAEAEKTFSYPLDLLLKLHDERVLVAFGQRDGIRVGHAVLAINGMDVNGRYTADGKEVLEYLGNPANYPVSIRFGRPRLTSNEKLMLASMFHSLFAIGSQLSPEQGSSGIEMLETDTFKLHCYQTLTGIKFVVLADPRQAGIDSLLRKIYEIYSDFALKNPFYSLEMPIRCELFDQNLKLALEVAEKAGTFGPGS",
# AT8A1_HUMAN
"MPTMRRTVSEIRSRAEGYEKTDDVSEKTSLADQEEVRTIFINQPQLTKFCNNHVSTAKYNIITFLPRFLYSQFRRAANSFFLFIALLQQIPDVSPTGRYTTLVPLLFILAVAAIKEIIEDIKRHKADNAVNKKQTQVLRNGAWEIVHWEKVAVGEIVKVTNGEHLPADLISLSSSEPQAMCYIETSNLDGETNLKIRQGLPATSDIKDVDSLMRISGRIECESPNRHLYDFVGNIRLDGHGTVPLGADQILLRGAQLRNTQWVHGIVVYTGHDTKLMQNSTSPPLKLSNVERITNVQILILFCILIAMSLVCSVGSAIWNRRHSGKDWYLNLNYGGASNFGLNFLTFIILFNNLIPISLLVTLEVVKFTQAYFINWDLDMHYEPTDTAAMARTSNLNEELGQVKYIFSDKTGTLTCNVMQFKKCTIAGVAYGHVPEPEDYGCSPDEWQNSQFGDEKTFSDSSLLENLQNNHPTAPIICEFLTMMAVCHTAVPEREGDKIIYQAASPDEGALVRAAKQLNFVFTGRTPDSVIIDSLGQEERYELLNVLEFTSARKRMSVIVRTPSGKLRLYCKGADTVIYDRLAETSKYKEITLKHLEQFATEGLRTLCFAVAEISESDFQEWRAVYQRASTSVQNRLLKLEESYELIEKNLQLLGATAIEDKLQDQVPETIETLMKADIKIWILTGDKQETAINIGHSCKLLKKNMGMIVINEGSLDGTRETLSRHCTTLGDALRKENDFALIIDGKTLKYALTFGVRQYFLDLALSCKAVICCRVSPLQKSEVVEMVKKQVKVVTLAIGDGANDVSMIQTAHVGVGISGNEGLQAANSSDYSIAQFKYLKNLLMIHGAWNYNRVSKCILYCFYKNIVLYIIEIWFAFVNGFSGQILFERWCIGLYNVMFTAMPPLTLGIFERSCRKENMLKYPELYKTSQNALDFNTKVFWVHCLNGLFHSVILFWFPLKALQYGTAFGNGKTSDYLLLGNFVYTFVVITVCLKAGLETSYWTWFSHIAIWGSIALWVVFFGIYSSLWPAIPMAPDMSGEAAMLFSSGVFWMGLLFIPVASLLLDVVYKVIKRTAFKTLVDEVQELEAKSQDPGAVVLGKSLTERAQLLKNVFKKNHVNLYRSESLQQNLLHGYAFSQDENGIVSQSEVIRAYDTTKQRPDEW",
# CSEN_HUMAN
"MQPAKEVTKASDGSLLGDLGHTPLSKKEGIKWQRPRLSRQALMRCCLVKWILSSTAPQGSDSSDSELELSTVRHQPEGLDQLQAQTKFTKKELQSLYRGFKNECPTGLVDEDTFKLIYAQFFPQGDATTYAHFLFNAFDADGNGAIHFEDFVVGLSILLRGTVHEKLKWAFNLYDINKDGYITKEEMLAIMKSIYDMMGRHTYPILREDAPAEHVERFFEKMDRNQDGVVTIEEFLEACQKDENIMSSMQLFENVI",
# FIS1_HUMAN
"MEAVLNELVSVEDLLKFEKKFQSEKAAGSVSKSTQFEYAWCLVRSKYNDDIRKGIVLLEELLPKGSKEEQRDYVFYLAVGNYRLKEYEKALKYVRGLLQTEPQNNQAKELERLIDKAMKKDGLVGMAIVGGMALGVAGLAGLIGLAVSKSKS",
# GOT1B_HUMAN
"MISLTDTQKIGMGLTGFGVFFLFFGMILFFDKALLAIGNVLFVAGLAFVIGLERTFRFFFQKHKMKATGFFLGGVFVVLIGWPLIGMIFEIYGFFLLFRGFFPVVVGFIRRVPVLGSLLNLPGIRSFVDKVGESNNMV",
# MYO5A_HUMAN
"MAASELYTKFARVWIPDPEEVWKSAELLKDYKPGDKVLLLHLEEGKDLEYHLDPKTKELPHLRNPDILVGENDLTALSYLHEPAVLHNLRVRFIDSKLIYTYCGIVLVAINPYEQLPIYGEDIINAYSGQNMGDMDPHIFAVAEEAYKQMARDERNQSIIVSGESGAGKTVSAKYAMRYFATVSGSASEANVEEKVLASNPIMESIGNAKTTRNDNSSRFGKYIEIGFDKRYRIIGANMRTYLLEKSRVVFQAEEERNYHIFYQLCASAKLPEFKMLRLGNADNFNYTKQGGSPVIEGVDDAKEMAHTRQACTLLGISESHQMGIFRILAGILHLGNVGFTSRDADSCTIPPKHEPLCIFCELMGVDYEEMCHWLCHRKLATATETYIKPISKLQATNARDALAKHIYAKLFNWIVDNVNQALHSAVKQHSFIGVLDIYGFETFEINSFEQFCINYANEKLQQQFNMHVFKLEQEEYMKEQIPWTLIDFYDNQPCINLIESKLGILDLLDEECKMPKGTDDTWAQKLYNTHLNKCALFEKPRLSNKAFIIQHFADKVEYQCEGFLEKNKDTVFEEQIKVLKSSKFKMLPELFQDDEKAISPTSATSSGRTPLTRTPAKPTKGRPGQMAKEHKKTVGHQFRNSLHLLMETLNATTPHYVRCIKPNDFKFPFTFDEKRAVQQLRACGVLETIRISAAGFPSRWTYQEFFSRYRVLMKQKDVLSDRKQTCKNVLEKLILDKDKYQFGKTKIFFRAGQVAYLEKLRADKLRAACIRIQKTIRGWLLRKKYLRMRKAAITMQRYVRGYQARCYAKFLRRTKAATIIQKYWRMYVVRRRYKIRRAATIVLQSYLRGFLARNRYRKILREHKAVIIQKRVRGWLARTHYKRSMHAIIYLQCCFRRMMAKRELKKLKIEARSVERYKKLHIGMENKIMQLQRKVDEQNKDYKCLVEKLTNLEGIYNSETEKLRSDLERLQLSEEEAKVATGRVLSLQEEIAKLRKDLEQTRSEKKCIEEHADRYKQETEQLVSNLKEENTLLKQEKEALNHRIVQQAKEMTETMEKKLVEETKQLELDLNDERLRYQNLLNEFSRLEERYDDLKEEMTLMVHVPKPGHKRTDSTHSSNESEYIFSSEIAEMEDIPSRTEEPSEKKVPLDMSLFLKLQKRVTELEQEKQVMQDELDRKEEQVLRSKAKEEERPQIRGAELEYESLKRQELESENKKLKNELNELRKALSEKSAPEVTAPGAPAYRVLMEQLTSVSEELDVRKEEVLILRSQLVSQKEAIQPKDDKNTMTDSTILLEDVQKMKDKGEIAQAYIGLKETNRSSALDYHELNEDGELWLVYEGLKQANRLLESQLQSQKRSHENEAEALRGEIQSLKEENNRQQQLLAQNLQLPPEARIEASLQHEITRLTNENLDLMEQLEKQDKTVRKLKKQLKVFAKKIGELEVGQMENISPGQIIDEPIRPVNIPRKEKDFQGMLEYKKEDEQKLVKNLILELKPRGVAVNLIPGLPAYILFMCVRHADYLNDDQKVRSLLTSTINSIKKVLKKRGDDFETVSFWLSNTCRFLHCLKQYSGEEGFMKHNTSRQNEHCLTNFDLAEYRQVLSDLAIQIYQQLVRVLENILQPMIVSGMLEHETIQGVSGVKPTGLRKRTSSIADEGTYTLDSILRQLNSFHSVMCQHGMDPELIKQVVKQMFYIIGAITLNNLLLRKDMCSWSKGMQIRYNVSQLEEWLRDKNLMNSGAKETLEPLIQAAQLLQVKKKTDDDAEAICSMCNALTTAQIVKVLNLYTPVNEFEERVSVSFIRTIQMRLRDRKDSPQLLMDAKHIFPVTFPFNPSSLALETIQIPASLGLGFISRV",
# TPPC1_HUMAN
"MTVHNLYLFDRNGVCLHYSEWHRKKQAGIPKEEEYKLMYGMLFSIRSFVSKMSPLDMKDGFLAFQTSRYKLHYYETPTGIKVVMNTDLGVGPIRDVLHHIYSALYVELVVKNPLCPLGQTVQSELFRSRLDSYVRSLPFFSARAG",
# S23IP_HUMAN
"MAERKPNGGSGGASTSSSGTNLLFSSSATEFSFNVPFIPVTQASASPASLLLPGEDSTDVGEEDSFLGQTSIHTSAPQTFSYFSQVSSSSDPFGNIGQSPLTTAATSVGQSGFPKPLTALPFTTGSQDVSNAFSPSISKAQPGAPPSSLMGINSYLPSQPSSLPPSYFGNQPQGIPQPGYNPYRHTPGSSRANPYIAPPQLQQCQTPGPPAHPPPSGPPVQMYQMPPGSLPPVPSSVQSPAQQQVPARPGAPSVQVPSPFLLQNQYEPVQPHWFYCKEVEYKQLWMPFSVFDSLNLEEIYNSVQPDPESVVLGTDGGRYDVYLYDRIRKAAYWEEEPAEVRRCTWFYKGDTDSRFIPYTEEFSEKLEAEYKKAVTTNQWHRRLEFPSGETIVMHNPKVIVQFQPSSVPDEWGTTQDGQTRPRVVKRGIDDNLDEIPDGEMPQVDHLVFVVHGIGPVCDLRFRSIIECVDDFRVVSLKLLRTHFKKSLDDGKVSRVEFLPVHWHSSLGGDATGVDRNIKKITLPSIGRFRHFTNETLLDILFYNSPTYCQTIVEKVGMEINHLHALFMSRNPDFKGGVSVAGHSLGSLILFDILSNQKDLNLSKCPGPLAVANGVVKQLHFQEKQMPEEPKLTLDESYDLVVENKEVLTLQETLEALSLSEYFSTFEKEKIDMESLLMCTVDDLKEMGIPLGPRKKIANFVEHKAAKLKKAASEKKAVAATSTKGQEQSAQKTKDMASLPSESNEPKRKLPVGACVSSVCVNYESFEVGAGQVSVAYNSLDFEPEIFFALGSPIAMFLTIRGVDRIDENYSLPTCKGFFNIYHPLDPVAYRLEPMIVPDLDLKAVLIPHHKGRKRLHLELKESLSRMGSDLKQGFISSLKSAWQTLNEFARAHTSSTQLQEELEKVANQIKEEEEKQVVEAEKVVESPDFSKDEDYLGKVGMLNGGRRIDYVLQEKPIESFNEYLFALQSHLCYWESEDTALLLLKEIYRTMNISPEQPQH"
]


# Add protein names corresponding to the sequences
protein_names = [
"TM129_HUMAN",
"ELOV7_HUMAN",
"NBAS_HUMAN",
"PEDS1_HUMAN",
"UBXN8_HUMAN",
"RN103_HUMAN",
"PGRC1_HUMAN",
"DNM1L_HUMAN",
"PLOD2_HUMAN",
"SCD_HUMAN",
"TOR1A_HUMAN",
"EI24_HUMAN",
"RGRF2_HUMAN",
"MGST3_HUMAN",
"CLGN_HUMAN",
"SC16A_HUMAN",
"PLCB_HUMAN",
"DEGS1_HUMAN",
"SPTC1_HUMAN",
"GPR37_HUMAN",
"INSI1_HUMAN",
"TPSN_HUMAN",
"ZW10_HUMAN",
"HTRA2_HUMAN",
"ERD23_HUMAN",
"G6PT1_HUMAN",
"CALU_HUMAN",
"CREB3_HUMAN",
"EXTL3_HUMAN",
"AT10A_HUMAN",
"MARH6_HUMAN",
"PLOD3_HUMAN",
"UD19_HUMAN",
"ICMT_HUMAN",
"DPM1_HUMAN",
"TMCC2_HUMAN",
"RTN2_HUMAN",
"PDCD6_HUMAN",
"ERN1_HUMAN",
"ERLN1_HUMAN",
"PR15A_HUMAN",
"SOAT2_HUMAN",
"WFS1_HUMAN",
"ERG24_HUMAN",
"AT10B_HUMAN",
"UFL1_HUMAN",
"ERLN2_HUMAN",
"SC31A_HUMAN",
"RTN3_HUMAN",
"STBD1_HUMAN",
"VAPB_HUMAN",
"SGPL1_HUMAN",
"ACSL3_HUMAN",
"FTCD_HUMAN",
"NB5R3_HUMAN",
"HMDH_HUMAN",
"RPN1_HUMAN",
"CP17A_HUMAN",
"SRPRA_HUMAN",
"STS_HUMAN",
"HMOX1_HUMAN",
"BCL2_HUMAN",
"MGST1_HUMAN",
"CP2D6_HUMAN",
"BIP_HUMAN",
"CP19A_HUMAN",
"PA2GA_HUMAN",
"ENPL_HUMAN",
"CD1D_HUMAN",
"AT2A2_HUMAN",
"XBP1_HUMAN",
"PTN1_HUMAN",
"ATF6A_HUMAN",
"UD16_HUMAN",
"AL5AP_HUMAN",
"UD11_HUMAN",
"UD14_HUMAN",
"3BHS2_HUMAN",
"PPLA_HUMAN",
"FKBP2_HUMAN",
"CERS1_HUMAN",
"CALR_HUMAN",
"CALX_HUMAN",
"PDIA3_HUMAN",
"HLAF_HUMAN",
"CASQ1_HUMAN",
"ERD22_HUMAN",
"PGH2_HUMAN",
"UD13_HUMAN",
"SOAT1_HUMAN",
"DHB3_HUMAN",
"FDFT_HUMAN",
"INGR2_HUMAN",
"OST48_HUMAN",
"SSRA_HUMAN",
"SSRB_HUMAN",
"CAMLG_HUMAN",
"FMO5_HUMAN",
"CASP4_HUMAN",
"TMEDA_HUMAN",
"PSN1_HUMAN",
"PSN2_HUMAN",
"LRBA_HUMAN",
"BAP31_HUMAN",
"PLAT3_HUMAN",
"BI1_HUMAN",
"TERA_HUMAN",
"TM50B_HUMAN",
"TMM33_HUMAN",
"TMPS3_HUMAN",
"SC61B_HUMAN",
"SPCS3_HUMAN",
"TM258_HUMAN",
"SELT_HUMAN",
"RL40_HUMAN",
"S35A2_HUMAN",
"AT11A_HUMAN",
"CAV1_HUMAN",
"TAP1_HUMAN",
"TAP2_HUMAN",
"UBXN1_HUMAN",
"PLP2_HUMAN",
"NMDZ1_HUMAN",
"KDSR_HUMAN",
"CKAP4_HUMAN",
"BAX_HUMAN",
"SCAP_HUMAN",
"SRBP2_HUMAN",
"ASPH_HUMAN",
"NMDE1_HUMAN",
"DLG1_HUMAN",
"SEC20_HUMAN",
"TRDN_HUMAN",
"CLN3_HUMAN",
"TMED1_HUMAN",
"PKD2_HUMAN",
"STIM1_HUMAN",
"MOGS_HUMAN",
"MLEC_HUMAN",
"FKBP8_HUMAN",
"PIGH_HUMAN",
"BECN1_HUMAN",
"ERG1_HUMAN",
"ITPR2_HUMAN",
"ITPR3_HUMAN",
"ITPR1_HUMAN",
"EMC2_HUMAN",
"HERP1_HUMAN",
"PDIA6_HUMAN",
"EBP_HUMAN",
"TMED2_HUMAN",
"DHC24_HUMAN",
"SC23B_HUMAN",
"TRAM1_HUMAN",
"NSDHL_HUMAN",
"MSMO1_HUMAN",
"EXT1_HUMAN",
"RPE65_HUMAN",
"PTGIS_HUMAN",
"RTN1_HUMAN",
"CP51A_HUMAN",
"LTC4S_HUMAN",
"T132A_HUMAN",
"MOGT2_HUMAN",
"GPAT3_HUMAN",
"ORML2_HUMAN",
"MEST_HUMAN",
"PIGG_HUMAN",
"S35D3_HUMAN",
"LRRK2_HUMAN",
"ZFY27_HUMAN",
"WLS_HUMAN",
"TSPO2_HUMAN",
"CDKAL_HUMAN",
"HACD4_HUMAN",
"LPCT4_HUMAN",
"CR3L3_HUMAN",
"SCAR3_HUMAN",
"ATLA3_HUMAN",
"MCTP1_HUMAN",
"RINT1_HUMAN",
"TM214_HUMAN",
"OFUT3_HUMAN",
"TXD11_HUMAN",
"LCTL_HUMAN",
"LCLT1_HUMAN",
"HACD2_HUMAN",
"CERS6_HUMAN",
"ALKMO_HUMAN",
"PLPL7_HUMAN",
"CR3L2_HUMAN",
"IKIP_HUMAN",
"TLC3B_HUMAN",
"PGAP1_HUMAN",
"NB5R4_HUMAN",
"FA2H_HUMAN",
"PCAT2_HUMAN",
"EPMIP_HUMAN",
"TMC6_HUMAN",
"SYVN1_HUMAN",
"LYRIC_HUMAN",
"GPAT4_HUMAN",
"KTN1_HUMAN",
"PIGZ_HUMAN",
"VRK2_HUMAN",
"ERO1B_HUMAN",
"LRC33_HUMAN",
"TMC8_HUMAN",
"CERS3_HUMAN",
"KSR1_HUMAN",
"TEX2_HUMAN",
"TRI59_HUMAN",
"SGPP2_HUMAN",
"PLPL6_HUMAN",
"ZDH14_HUMAN",
"VKORL_HUMAN",
"ORML3_HUMAN",
"ARM10_HUMAN",
"UB2J2_HUMAN",
"TMTC2_HUMAN",
"RDHE2_HUMAN",
"VMA12_HUMAN",
"CERS5_HUMAN",
"CISD2_HUMAN",
"JAGN1_HUMAN",
"PAR16_HUMAN",
"SERP2_HUMAN",
"AN13C_HUMAN",
"ZDH22_HUMAN",
"PNDC1_HUMAN",
"PMGT2_HUMAN",
"AT11C_HUMAN",
"UBAC2_HUMAN",
"AQP11_HUMAN",
"B3GL2_HUMAN",
"PCAT1_HUMAN",
"TOIP2_HUMAN",
"ATLA2_HUMAN",
"NPL4_HUMAN",
"HMGC2_HUMAN",
"DJB14_HUMAN",
"CPT1C_HUMAN",
"STT3B_HUMAN",
"SPP_HUMAN",
"MRAP_HUMAN",
"ACER1_HUMAN",
"RHBL4_HUMAN",
"CR3L4_HUMAN",
"ZDHC1_HUMAN",
"RN139_HUMAN",
"ZFN2B_HUMAN",
"APH1B_HUMAN",
"GIMA1_HUMAN",
"ATLA1_HUMAN",
"RSAD2_HUMAN",
"S39A7_HUMAN",
"PIEZ1_HUMAN",
"NICA_HUMAN",
"UBXN4_HUMAN",
"EDEM1_HUMAN",
"SORL_HUMAN",
"UFD1_HUMAN",
"CDS1_HUMAN",
"EXT2_HUMAN",
"AT2A3_HUMAN",
"HMR1_HUMAN",
"FATE1_HUMAN",
"YIPF5_HUMAN",
"S35B4_HUMAN",
"TM203_HUMAN",
"CCD47_HUMAN",
"LRC59_HUMAN",
"CR3L1_HUMAN",
"APH1A_HUMAN",
"SARAF_HUMAN",
"TBC20_HUMAN",
"FAF2_HUMAN",
"RN183_HUMAN",
"CERS2_HUMAN",
"MRAP2_HUMAN",
"VMP1_HUMAN",
"RN185_HUMAN",
"ZDH12_HUMAN",
"S39AD_HUMAN",
"ERO1A_HUMAN",
"REEP6_HUMAN",
"DDRGK_HUMAN",
"DNJC1_HUMAN",
"SAMD8_HUMAN",
"DGAT2_HUMAN",
"SIM14_HUMAN",
"PANX1_HUMAN",
"CLCC1_HUMAN",
"CP2S1_HUMAN",
"SEC62_HUMAN",
"GPER1_HUMAN",
"PMYT1_HUMAN",
"SGMR1_HUMAN",
"MGST2_HUMAN",
"RNF5_HUMAN",
"PLCA_HUMAN",
"SELS_HUMAN",
"REEP2_HUMAN",
"SC22C_HUMAN",
"ADPGK_HUMAN",
"ESYT1_HUMAN",
"ALG1_HUMAN",
"G6PC3_HUMAN",
"DERL1_HUMAN",
"ALG12_HUMAN",
"TMED9_HUMAN",
"ELOV1_HUMAN",
"EBPL_HUMAN",
"UDB28_HUMAN",
"LNP_HUMAN",
"DERL2_HUMAN",
"ELOV4_HUMAN",
"MAGT1_HUMAN",
"UN93B_HUMAN",
"PORCN_HUMAN",
"ELOV6_HUMAN",
"REEP4_HUMAN",
"RETR1_HUMAN",
"ZDHC6_HUMAN",
"ALG9_HUMAN",
"SR5A3_HUMAN",
"ZDH11_HUMAN",
"REEP1_HUMAN",
"CERS4_HUMAN",
"UD17_HUMAN",
"UD110_HUMAN",
"UD18_HUMAN",
"ELOV3_HUMAN",
"ZFYV1_HUMAN",
"RDH14_HUMAN",
"HHATL_HUMAN",
"ABCB6_HUMAN",
"ISK5_HUMAN",
"RTN4_HUMAN",
"TLR9_HUMAN",
"PLCC_HUMAN",
"CLN6_HUMAN",
"MARH5_HUMAN",
"ELOV2_HUMAN",
"NSMA3_HUMAN",
"DJB12_HUMAN",
"TLR7_HUMAN",
"ELOV5_HUMAN",
"TECR_HUMAN",
"ERAP1_HUMAN",
"PEN2_HUMAN",
"USE1_HUMAN",
"E2AK3_HUMAN",
"BFAR_HUMAN",
"CNIH4_HUMAN",
"HACD3_HUMAN",
"VAPA_HUMAN",
"MARH2_HUMAN",
"ORML1_HUMAN",
"AT10D_HUMAN",
"STIM2_HUMAN",
"RRBP1_HUMAN",
"KLH14_HUMAN",
"DISP3_HUMAN",
"STX18_HUMAN",
"DPM3_HUMAN",
"DHCR7_HUMAN",
"SPAST_HUMAN",
"EXTL2_HUMAN",
"DNJB9_HUMAN",
"SE1L1_HUMAN",
"CLN8_HUMAN",
"SEC63_HUMAN",
"NAT8_HUMAN",
"ZDHC2_HUMAN",
"MA1B1_HUMAN",
"AMFR_HUMAN",
"DSE_HUMAN",
"TMCO1_HUMAN",
"PHTF1_HUMAN",
"BOK_HUMAN",
"S27A5_HUMAN",
"TMM98_HUMAN",
"ZDHC9_HUMAN",
"TMED7_HUMAN",
"TMED3_HUMAN",
"GRIP1_HUMAN",
"TBL2_HUMAN",
"IR3IP_HUMAN",
"PEX16_HUMAN",
"UBIA1_HUMAN",
"POMT1_HUMAN",
"CP46A_HUMAN",
"SELK_HUMAN",
"SERP1_HUMAN",
"BNIP3_HUMAN",
"AG10B_HUMAN",
"S27A3_HUMAN",
"S27A1_HUMAN",
"SMS1_HUMAN",
"TMX4_HUMAN",
"SAR1A_HUMAN",
"ZNT1_HUMAN",
"EST2_HUMAN",
"TOR1B_HUMAN",
"P4HA2_HUMAN",
"CRTAP_HUMAN",
"STC2_HUMAN",
"A1AT_HUMAN",
"APOE_HUMAN",
"ALBU_HUMAN",
"VTNC_HUMAN",
"PROC_HUMAN",
"PDIA1_HUMAN",
"TSP1_HUMAN",
"CO1A2_HUMAN",
"CLUS_HUMAN",
"PDIA4_HUMAN",
"P4HA1_HUMAN",
"GLU2B_HUMAN",
"EST1_HUMAN",
"PPIB_HUMAN",
"ERP29_HUMAN",
"AMRP_HUMAN",
"FLT3_HUMAN",
"SERPH_HUMAN",
"MANF_HUMAN",
"MTP_HUMAN",
"CO4A3_HUMAN",
"PDIA2_HUMAN",
"DNJC3_HUMAN",
"RCN2_HUMAN",
"RCN1_HUMAN",
"P3H1_HUMAN",
"P3H2_HUMAN",
"DJC10_HUMAN",
"SUMF2_HUMAN",
"SUMF1_HUMAN",
"PCSK9_HUMAN",
"TXND5_HUMAN",
"SELM_HUMAN",
"ARSG_HUMAN",
"GPX7_HUMAN",
"EDEM2_HUMAN",
"EDEM3_HUMAN",
"SIL1_HUMAN",
"TOR3A_HUMAN",
"UGGG2_HUMAN",
"UGGG1_HUMAN",
"CATZ_HUMAN",
"DJB11_HUMAN",
"MINP1_HUMAN",
"HYOU1_HUMAN",
"CC88B_HUMAN",
"U17L3_HUMAN",
"U17L4_HUMAN",
"U17L5_HUMAN",
"U17LF_HUMAN",
"U17LA_HUMAN",
"U17LD_HUMAN",
"UL17C_HUMAN",
"U17LB_HUMAN",
"U17LL_HUMAN",
"U17LI_HUMAN",
"U17LM_HUMAN",
"U17LH_HUMAN",
"U17LJ_HUMAN",
"U17LK_HUMAN",
"ST2B1_HUMAN",
"PDE2A_HUMAN",
"P3C2B_HUMAN",
"PDZD2_HUMAN",
"YKT6_HUMAN",
"AT8B1_HUMAN",
"TPPC3_HUMAN",
"NCK2_HUMAN",
"PLIN1_HUMAN",
"PRKN_HUMAN",
"CUBN_HUMAN",
"HNRPQ_HUMAN",
"USO1_HUMAN",
"DIAP2_HUMAN",
"CLN5_HUMAN",
"TPC6A_HUMAN",
"PDE9A_HUMAN",
"SRP72_HUMAN",
"BAIP3_HUMAN",
"AHSA1_HUMAN",
"PSMG1_HUMAN",
"STAU1_HUMAN",
"G6B_HUMAN",
"OAS1_HUMAN",
"FOS_HUMAN",
"CATA_HUMAN",
"PRIO_HUMAN",
"P53_HUMAN",
"APOD_HUMAN",
"RLA0_HUMAN",
"IFIT2_HUMAN",
"U17L7_HUMAN",
"U17L8_HUMAN",
"TPC2A_HUMAN",
"HS71A_HUMAN",
"HS71B_HUMAN",
"NCK1_HUMAN",
"KPCA_HUMAN",
"CAN2_HUMAN",
"PTN2_HUMAN",
"ICAL_HUMAN",
"ACOHC_HUMAN",
"TGM2_HUMAN",
"RS3_HUMAN",
"EF1B_HUMAN",
"EF1G_HUMAN",
"RL10_HUMAN",
"RABP2_HUMAN",
"EF1D_HUMAN",
"OAS2_HUMAN",
"AL3A1_HUMAN",
"S10A7_HUMAN",
"MA1A1_HUMAN",
"MP2K2_HUMAN",
"VHL_HUMAN",
"HD_HUMAN",
"CACP_HUMAN",
"RL27A_HUMAN",
"HSP13_HUMAN",
"EMD_HUMAN",
"GDS1_HUMAN",
"HAP1_HUMAN",
"EPHB1_HUMAN",
"TCL1A_HUMAN",
"RAB38_HUMAN",
"UB2G2_HUMAN",
"RS3A_HUMAN",
"RS8_HUMAN",
"CNBP_HUMAN",
"UBX2A_HUMAN",
"DLG4_HUMAN",
"NUCB2_HUMAN",
"TMF1_HUMAN",
"SET_HUMAN",
"KPCE_HUMAN",
"MP2K1_HUMAN",
"ATP7A_HUMAN",
"KPCD_HUMAN",
"RL18_HUMAN",
"B2CL1_HUMAN",
"VAC14_HUMAN",
"KCNA1_HUMAN",
"U17LO_HUMAN",
"WASC5_HUMAN",
"HYAL2_HUMAN",
"AIMP1_HUMAN",
"ANK3_HUMAN",
"RHG05_HUMAN",
"PDE3B_HUMAN",
"SQSTM_HUMAN",
"RAB32_HUMAN",
"KEAP1_HUMAN",
"UBX2B_HUMAN",
"MLC1_HUMAN",
"BAK_HUMAN",
"GRDN_HUMAN",
"CERKL_HUMAN",
"TPC3L_HUMAN",
"MCTP2_HUMAN",
"ANO7_HUMAN",
"CAVN1_HUMAN",
"KCIP4_HUMAN",
"CASPC_HUMAN",
"NHLC1_HUMAN",
"GGYF2_HUMAN",
"NBEL2_HUMAN",
"FGD5_HUMAN",
"UN13D_HUMAN",
"U17L1_HUMAN",
"TPC6B_HUMAN",
"TRXR3_HUMAN",
"HPS6_HUMAN",
"TPPC5_HUMAN",
"MSRB3_HUMAN",
"ABI1_HUMAN",
"GIMA8_HUMAN",
"PXL2B_HUMAN",
"RN128_HUMAN",
"PSMF1_HUMAN",
"KIFA3_HUMAN",
"CIDEC_HUMAN",
"ACBG1_HUMAN",
"SAHH3_HUMAN",
"AIFM3_HUMAN",
"NLRP3_HUMAN",
"TPPC9_HUMAN",
"SRPK1_HUMAN",
"PARK7_HUMAN",
"PLIN2_HUMAN",
"INP5K_HUMAN",
"S26A6_HUMAN",
"SCAPE_HUMAN",
"ULBP1_HUMAN",
"LSG1_HUMAN",
"5NT3A_HUMAN",
"GBRL1_HUMAN",
"TMT1A_HUMAN",
"DISC1_HUMAN",
"CS012_HUMAN",
"DUS2_HUMAN",
"PK2L1_HUMAN",
"SYLC_HUMAN",
"SRP68_HUMAN",
"UBP25_HUMAN",
"DDX41_HUMAN",
"FBX2_HUMAN",
"TPC2L_HUMAN",
"ASC_HUMAN",
"UBQL1_HUMAN",
"CHIP_HUMAN",
"ANGE1_HUMAN",
"POMP_HUMAN",
"TPPC4_HUMAN",
"AT8A1_HUMAN",
"CSEN_HUMAN",
"FIS1_HUMAN",
"GOT1B_HUMAN",
"MYO5A_HUMAN",
"TPPC1_HUMAN",
"S23IP_HUMAN"

]

In [74]:
scores = predict_condensates(model, sequences, batch_size=1)

100%|█████████████████████████████████████████████████████████████| 628/628 [00:06<00:00, 96.76it/s]


In [75]:
data = {"Protein": protein_names, "sequences": sequences}
for j, condensate in enumerate(COMPARTMENT_CLASSES):
    data[f"{condensate.upper()}_Score"] = scores[:, j].tolist()

In [76]:
pd.DataFrame(data)

,Protein,sequences,CYTOSOL_Score,ER_Score,MITOCHONDRION_Score,NUCLEUS_Score,EXCITATORY SYNAPSE_Score,INHIBITORY SYNAPSES_Score
0,TM129_HUMAN,MDSPEVTFTLAYLVFAVCFVFTPNEFHAAGLTVQNLLSGWLGSEDA...,0.023,0.830,0.004,0.015,0.060,0.061
1,ELOV7_HUMAN,MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...,0.011,0.945,0.006,0.031,0.003,0.007
2,NBAS_HUMAN,MAAPESGPALSPGTAEGEEETILYDLLVNTEWPPETEVQPRGNQKH...,0.823,0.002,0.010,0.020,0.068,0.178
3,PEDS1_HUMAN,MAGAENWPGQQLELDEDEASCCRWGAQHAGARELAALYSPGKRLQE...,0.918,0.003,0.004,0.060,0.014,0.033
4,UBXN8_HUMAN,MASRGVVGIFFLSAVPLVCLELRRGIPDIGIKDFLLLCGRILLLLA...,0.091,0.573,0.002,0.029,0.132,0.111
...,...,...,...,...,...,...,...,...
623,FIS1_HUMAN,MEAVLNELVSVEDLLKFEKKFQSEKAAGSVSKSTQFEYAWCLVRSK...,0.200,0.012,0.674,0.009,0.031,0.158
624,GOT1B_HUMAN,MISLTDTQKIGMGLTGFGVFFLFFGMILFFDKALLAIGNVLFVAGL...,0.013,0.893,0.021,0.016,0.014,0.037
625,MYO5A_HUMAN,MAASELYTKFARVWIPDPEEVWKSAELLKDYKPGDKVLLLHLEEGK...,0.041,0.001,0.015,0.827,0.018,0.036
626,TPPC1_HUMAN,MTVHNLYLFDRNGVCLHYSEWHRKKQAGIPKEEEYKLMYGMLFSIR...,0.523,0.002,0.050,0.346,0.024,0.073


In [77]:
# Your data (assuming it's already available as `data`)
df = pd.DataFrame(data)

# Prompt user to choose save location and filename
Tk().withdraw()  # Hide the root window
save_path = filedialog.asksaveasfilename(
    defaultextension=".xlsx",
    filetypes=[("Excel files", "*.xlsx")],
    title="Save predictions as...",
    initialfile="Predictions.xlsx"  # Default filename suggestion
)

# Save to Excel
if save_path:  # Only save if user didn't cancel
    df.to_excel(save_path, index=False)